In [ ]:
%%time

from subprocess import Popen, PIPE
from math import *
import random
import numpy as np
import random as pr
from copy import deepcopy

import itertools
import time
import math
import argparse
import subprocess
from load_model import loaded_model
from keras.preprocessing import sequence
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
from rdkit.Chem import MolFromSmiles, MolToSmiles

from rdkit import Chem
from rdkit.Chem import RDConfig
import os
import sys
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))

import sascorer

from make_smile import zinc_data_with_bracket_original, zinc_processed_with_bracket
from add_node_type4_SA import chem_kn_simulation, make_input_smile,predict_smile,check_node_type,node_to_add,expanded_node

''' !add_node_type4_SA! '''

class chemical:

    def __init__(self):

        self.position=['&']
    def Clone(self):

        st = chemical()
        st.position= self.position[:]
        return st

    def SelectPosition(self,m):

        self.position.append(m)

    def Getatom(self):
        return [i for i in range(self.num_atom)]

class Node:

    def __init__(self, position = None,  parent = None, state = None):
        self.position = position
        self.parentNode = parent
        self.childNodes = []
        self.child=None
        self.wins = 0
        self.visits = 0
        #self.nonvisited_atom=state.Getatom()
        #self.type_node=tp
        self.depth=0


    def Selectnode(self):
        ucb=[]
        for i in range(len(self.childNodes)):
            ucb.append(self.childNodes[i].wins/self.childNodes[i].visits+1.0*sqrt(2*log(self.visits)/self.childNodes[i].visits))
        m = np.amax(ucb)
        #print('ucb', ucb)
        #print('m', m)   ###
        indices = np.nonzero(ucb == m)[0]
        
        #print('inbdices', indices)
        ind=pr.choice(indices)
        s=self.childNodes[ind]
        return s

    def Addnode(self, m, s):

        n = Node(position = m, parent = self, state = s)
        self.childNodes.append(n)

    def simulation(self,state):
        predicted_smile=predict_smile(model,state)
        input_smile=make_input_smile(predicted_smile)
        logp,valid_smile,all_smile=logp_calculation(input_smile)

        return logp,valid_smile,all_smile

    def Update(self, result):

        self.visits += 1
        self.wins += result

def MCTS(root, verbose = False):

    All_SA_list=[]
    All_cycle_list=[]
    #All_MolLogP_list=[]
    #All_MolWt_list=[]
    All_mobility_list=[]
    All_score_list =[]
    
    """initialization of the chemical trees and grammar trees"""
    run_time=time.time()+600*2
    rootnode = Node(state = root)
    state = root.Clone()
    maxnum=0
    iteration_num=0
    start_time=time.time()
    """----------------------------------------------------------------------"""

    """global variables used for save valid compounds and simulated compounds"""
    valid_compound=[]
    all_simulated_compound=[]
    desired_compound=[]
    max_score=-100.0
    desired_activity=[]
    time_distribution=[]
    num_searched=[]
    current_score=[]
    depth=[]
    all_score=[]


    """----------------------------------------------------------------------"""

    while maxnum < 10:
        print(maxnum)
        node = rootnode
        state = root.Clone()
        """selection step"""
        node_pool=[]
        print("current found max_score:",max_score)

        while node.childNodes!=[]:
            node = node.Selectnode()
            state.SelectPosition(node.position)
        print("state position:",state.position)
        depth.append(len(state.position))
        print(len(state.position))
        if len(state.position)>=81:
            re=-1.0
            while node != None:
                node.Update(re)
                node = node.parentNode
        else:
            """------------------------------------------------------------------"""

            """expansion step"""
            """calculate how many nodes will be added under current leaf"""
            expanded=expanded_node(model,state.position,val)
            nodeadded=node_to_add(expanded,val)


            all_posible=chem_kn_simulation(model,state.position,val,nodeadded)
            generate_smile=predict_smile(all_posible,val)
            new_compound=make_input_smile(generate_smile)

            node_index,score,valid_smile,all_smile,mobility_list=check_node_type(new_compound,SA_mean,SA_std,logP_mean,logP_std,cycle_mean,cycle_std)
            
            print('node_index', node_index)
            
            All_mobility_list.extend(mobility_list) 
            All_score_list.extend(score)
            
            print("node_index:", node_index)
            valid_compound.extend(valid_smile)
            all_simulated_compound.extend(all_smile)
            
            all_score.extend(score)
            iteration_num=len(all_simulated_compound)
            if len(node_index)==0:
                re=-1.0
                while node != None:
                    node.Update(re)
                    node = node.parentNode
            else:
                re=[]
                for i in range(len(node_index)):
                    m=node_index[i]
                    maxnum=maxnum+1
                    node.Addnode(nodeadded[m],state)
                    node_pool.append(node.childNodes[i])
                    if score[i]>=max_score:
                        max_score=score[i]
                        current_score.append(max_score)
                    else:
                        current_score.append(max_score)
                    depth.append(len(state.position))
                    """simulation"""
                    re.append((0.8*score[i])/(1.0+abs(0.8*score[i])))
                    if maxnum==100:
                        maxscore100=max_score
                        time100=time.time()-start_time
                    if maxnum==500:
                        maxscore500=max_score
                        time500=time.time()-start_time
                    if maxnum==1000:

                        maxscore1000=max_score
                        time1000=time.time()-start_time
                    if maxnum==5000:
                        maxscore5000=max_score
                        time5000=time.time()-start_time
                    if maxnum==10000:
                        time10000=time.time()-start_time
                        maxscore10000=max_score

                    """backpropation step"""


                for i in range(len(node_pool)):

                    node=node_pool[i]
                    while node != None:
                        node.Update(re[i])
                        node = node.parentNode

        """check if found the desired compound"""

    finished_run_time=time.time()-start_time

    print("logp max found:", current_score)
    
    print('')
    print('All_mobility_list', All_mobility_list)
    print('All_score_list= ', All_score_list)
    print('')
    print("valid_com=",valid_compound)
    print("num_valid:", len(valid_compound))
    print("num_all_valid:", len(all_simulated_compound))
  
   
    dict_from_list = dict(zip(All_mobility_list, valid_compound))   
    result_list2 = [i for _,i in sorted(zip(All_mobility_list, valid_compound),reverse=True)]   
    
    result_list3 = [i for _,i in sorted(zip(All_score_list, valid_compound),reverse=True)]   
    
    
    print("sorted_smiles=",result_list2)
    print('length= ', len(result_list2))
    print('#########')
    
    
    print("sorted_smiles_score=",result_list3)
    print('length= ', len(result_list3))
    print('#########')
    
    print('len(depth)=',len(depth))
    print("runtime",finished_run_time)

    return valid_compound


def UCTchemical():
    one_search_start_time=time.time()
    time_out=one_search_start_time+60*10
    state = chemical()
    best = MCTS(root = state,verbose = False)


    return best


if __name__ == "__main__":
    smile_old=zinc_data_with_bracket_original()
    val,smile=zinc_processed_with_bracket(smile_old)
    print(val)
    #val=['\n', '&', 'C', '(', ')', 'c', '1', '2', 'o', '=', 'O', 'N', '3', 'F', '[C@@H]', 'n', '#', 'S', 'Cl', '[O-]', '[C@H]', '[NH+]', '[C@]', 's', 'Br' '[nH]', '[NH3+]', '4', '[NH2+]', '[C@@]', '[N+]', '[nH+]', '[N-]', '[n+]', 'I', '[n-]', '[OH+]', '[NH-]', '[o+]', '[CH2-]', '[CH-]', '[O+]']

    logP_values = np.loadtxt('logP_values.txt')
    SA_scores = np.loadtxt('SA_scores.txt')
    cycle_scores = np.loadtxt('cycle_scores.txt')
    SA_mean =  np.mean(SA_scores)
    print(len(SA_scores))

    SA_std=np.std(SA_scores)
    logP_mean = np.mean(logP_values)
    logP_std= np.std(logP_values)
    cycle_mean = np.mean(cycle_scores)
    cycle_std=np.std(cycle_scores)
    #val2=['C', '(',  'c', '1',  'o', '=', 'O', 'N', 'F', '[C@@H]', 'n',  'S', 'Cl', '[O-]']
    #val2=['C', 'c','#', '3', '(', '2', 'n', 'O', '/', 'N', '=', '\\', ')', '1', 'o', '4', 's', '[C@H]', 'F', 'S', 'Cl', '[C@@H]', '[C@@]', '[C@]', '5', '#', '[nH]', 'Br', 'I', '6', '-', '[NH+]', '[N-]', '[N+]', '[n+]', '[nH+]', '[NH2+]','[NH3+]']
    #val=['\n', '&', 'C', '(', ')', 'c', '1', '2', 'o', '=', 'O', 'N', '3', 'F', '[C@@H]', 'n', '#', 'S', 'Cl', '[O-]', '[C@H]>
    val2=['C', '(', ')', 'c', '1', '2', 'o', '=', 'O', 'N', '3', 'F', '[C@@H]', 'n', '-', '#', 'S', 'Cl', '[O-]', '[C@H]', '[NH+]', '[C@]', 's', 'Br', '/', '[nH]', '[NH3+]', '4', '[NH2+]', '[C@@]', '[N+]', '[nH+]', '\\', '[S@]', '5', '[N-]', '[n+]', '[S@@]', '[S-]', '6', '7', 'I', '[n-]', 'P', '[OH+]', '[NH-]', '[P@@H]', '[P@@]', '[PH2]', '[P@]', '[P+]', '[S+]', '[o+]', '[CH2-]', '[CH-]', '[SH+]', '[O+]', '[s+]', '[PH+]', '[PH]', '8', '[S@@+]']

    model=loaded_model()
    #acitivity_model=loaded_activity_model()
    valid_compound=UCTchemical()
    

In [ ]:
All_mobility_list= [1.3978653960564194e-06, 1.2202974482280876e-06, 2.226092058836916e-06, 1.1692704369003209e-05, 7.148841184995432e-07, 3.135956363593708e-06, 1.3879651299225516e-06, 4.2282731206903595e-06, 4.077534741413203e-07, 2.2062776012554765e-06, 2.736338890755746e-06, 1.0596986464595333e-06, 2.1525954613565453e-06, 0, 3.268244457771053e-06, 5.659443241811245e-06, 2.735868244168099e-06, 4.067538640356365e-06, 1.933019786030831e-06, 2.4869162229362467e-06, 6.066604634407486e-06, 3.853548082062379e-06, 1.704385613224104e-06, 1.5800663632424672e-06, 2.089838879555999e-05, 3.128313962420192e-06, 3.6399712468961155e-06, 3.281278804592368e-06, 5.1852493793228835e-06, 2.372886853635971e-06, 8.696944249932685e-06, 3.2992426583683785e-06, 3.942501766726208e-06, 0, 3.690513254820036e-06, 3.0087676702090593e-06, 5.839570680072025e-06, 1.1216146731775124e-06, 6.315435300896356e-06, 1.4241426706598351e-06, 4.513420074165288e-06, 5.1063165776927036e-06, 5.51058213728053e-06, 3.854300574483933e-06, 2.022671658134908e-06, 2.418648856094742e-06, 4.905854878187475e-06, 1.6933607938075226e-06, 2.504952293159863e-06, 3.825813334330232e-06, 2.562232972823325e-06, 4.256825868942761e-06, 7.906026856244054e-06, 3.0101381551336006e-06, 1.1849211394022841e-06, 3.2871019366042166e-06, 6.8894667659849785e-06, 3.9776803418077575e-06, 1.0298062574921609e-06, 4.0356959646317065e-06, 4.181746345015836e-06, 2.5306653586664837e-06, 1.3979028498848765e-06, 3.0986200506853703e-06, 1.1514715617612694e-06, 1.4670694919227847e-06, 1.2382156417892652e-06, 2.665778327619602e-06, 7.275965102626828e-06, 1.838980667076299e-06, 1.618048570301239e-06, 2.0013741596837288e-06, 1.4767485611591908e-06, 3.464137277501801e-06, 2.9907543199982036e-06, 3.0051663958740497e-06, 2.7482974473174037e-06, 3.3848954117864607e-06, 1.8622953777191926e-06, 5.768649236014715e-06, 6.500180525726807e-06, 1.9503966669415953e-06, 1.1183883479245031e-05, 2.8207617398152798e-06, 1.8116108196785597e-06, 2.1484858860874347e-06, 1.1340960292297039e-06, 1.4683537118039638e-06, 1.477137010789849e-05, 3.807101075918121e-06, 1.941879097963364e-06, 3.4385280293853e-06, 2.481627205942205e-06, 4.041068216558601e-06, 5.03793830708954e-06, 0, 2.5525641966112055e-06, 1.0708400309217562e-06, 2.4711939084438885e-06, 4.12891124555466e-06, 5.211909211708822e-06, 1.3989921996113154e-06, 0, 1.2360381556996646e-05, 2.8063382255411827e-06, 1.1505503847291512e-06, 4.300457525241699e-06, 5.690478953290663e-06, 1.7947631366041315e-06, 6.883025481448555e-06, 2.135605937320683e-06, 1.0663271255157053e-06, 5.985740376801763e-06, 1.582594758576325e-06, 4.5287909686489535e-06, 2.987783160944788e-06, 8.374412592843202e-06, 3.0674216907059873e-06, 2.0233662427848943e-06, 1.95850836270701e-06, 2.3084870904258153e-06, 2.7149758716515577e-06, 2.3586543729136225e-06, 1.8433885284875014e-06, 9.793849639060748e-07, 1.559095374934028e-06, 4.607271764273824e-06, 9.12966624129071e-07, 1.1690866703770293e-06, 2.587196483987095e-06, 1.5920477403801108e-06, 3.8431044937942854e-06, 0, 2.3362728637889255e-06, 9.477437478960946e-06, 0, 3.075718159598447e-06, 3.5350743121742506e-06, 3.815457303699005e-06, 1.0311122713543064e-06, 2.253897912329749e-06, 1.1928214443248663e-06, 2.097629006290571e-06, 2.4097588531706305e-06, 3.0066919526831668e-06, 8.734107809832211e-06, 3.2324386624230472e-06, 2.5953427257025683e-06, 2.0646020602079173e-06, 3.9697401677961246e-06, 8.206159606148897e-07, 2.4918463724769055e-06, 2.5030330673169993e-06, 3.506400252113629e-06, 7.950371203013894e-06, 0, 4.062885388404528e-06, 3.947997273081202e-06, 3.990011029436911e-06, 3.939634493775441e-06, 1.1710971548382179e-05, 2.649409545834492e-06, 1.3409766637647578e-06, 2.4318271556204297e-06, 4.91238951650961e-06, 2.0942926983212457e-06, 1.3509916539870157e-06, 5.690401115306968e-06, 4.3108219117522015e-06, 0, 5.060100939950887e-06, 0, 6.0823036937546944e-06, 1.2590488141383467e-06, 8.928519768167081e-07, 3.0677156008887444e-06, 8.127996646191382e-07, 1.4836490419118373e-06, 3.3055388073836706e-06, 6.108669665396685e-06, 3.299267260799275e-06, 3.910822776283813e-06, 4.2088046298914946e-06, 5.143647284499887e-06, 5.011877090105202e-06, 1.2176811792576181e-05, 2.119892281793015e-06, 1.6463874927365038e-06, 5.6204411254873036e-06, 1.054680540555444e-06, 1.657375987332007e-06, 5.275683749678796e-06, 1.388272108282757e-06, 3.730159720928535e-06, 1.0543184826430708e-05, 4.6661233311692475e-06, 2.3673477382580875e-06, 0, 2.2121067257606154e-06, 3.2789422808023703e-06, 4.625927014964619e-06, 1.223554143850017e-06, 1.0773956961647488e-05, 2.4111226049938957e-06, 3.3848954117864607e-06, 1.7774324540068012e-06, 1.7827271721503477e-06, 1.911562408800769e-06, 3.6666085137825738e-06, 4.306508730975103e-06, 3.8952626507532304e-06, 2.6128642133628936e-06, 2.8429735750440962e-06, 3.4885116905539888e-06, 5.5121280602724234e-06, 4.026553806544347e-06, 3.7627919865912537e-06, 3.576925741850015e-06, 1.604152945258584e-06, 8.702644920379892e-06, 4.825999651459529e-06, 4.2531572064363035e-06, 1.6394764240572105e-06, 4.18135437411448e-06, 3.266060544294499e-06, 1.657128977468648e-06, 4.304508899501047e-06, 2.3332282233805295e-06, 3.555781692629342e-06, 7.0724318452242186e-06, 3.9271372264598146e-05, 2.933566302544341e-06, 2.2634105489692674e-06, 2.27751882671737e-06, 1.9576885303291786e-06, 2.234394890066664e-06, 4.5583106586240705e-06, 2.2227056114662917e-06, 4.93247532597993e-06, 1.1404414243728961e-06, 1.5540647619542362e-06, 3.234983863027134e-06, 6.828016535710023e-06, 1.386700592418293e-06, 5.328471348558376e-06, 2.0991522338914573e-06, 7.930705779496866e-06, 1.5189123335368157e-05, 1.1162907584807783e-05, 1.7012207855948874e-06, 2.4452937259752495e-05, 1.1751384835253573e-06, 9.589048069669678e-07, 1.1597327897624424e-06, 9.056824042257886e-06, 8.180349033932066e-06, 4.740127879314944e-06, 4.469912815346155e-06, 2.4858728106060517e-06, 2.511281686731035e-06, 1.2715518175742691e-06, 3.2481361012617025e-06, 6.477383324901055e-06, 3.1722241001155057e-06, 1.9770089151155366e-05, 4.588510654302662e-06, 9.483889554213167e-07, 2.5587096847426963e-05, 1.8431275119079894e-06, 1.5061185148971365e-06, 2.127696900879051e-06, 3.527463947145928e-06, 6.123006042724284e-06, 0, 6.115995932861352e-06, 3.7778927069616122e-06, 3.487604626846932e-06, 0, 0, 5.53896753553951e-06, 4.442192953688894e-06, 1.7089969058290212e-06, 6.156527077539654e-06, 3.2597077605930604e-06, 1.4836691781072654e-06, 2.110152772752034e-06, 4.469086749203598e-06, 1.7701720123742627e-06, 3.3991429401429352e-06, 5.652391918596226e-06, 5.71246422644896e-06, 2.585727622929386e-06, 1.8258773002746345e-06, 5.100594203935559e-06, 2.18929027638125e-06, 2.5928153527025217e-06, 2.5911537479608076e-06, 2.064808484752522e-06, 3.174487482716841e-06, 1.9992211096167646e-06, 3.1669847247170926e-06, 1.543266558430881e-06, 1.6972025633618102e-06, 4.991416108860322e-07, 1.1660555862014098e-06, 1.6478877088851337e-05, 1.0918197340455057e-06, 3.5822550922729828e-06, 1.0824414397357107e-06, 0, 1.8599943295094086e-06, 3.425379481927112e-06, 1.3684366393198965e-06, 7.182818242900112e-06, 2.8751148979189473e-06, 1.0829061342889164e-06, 2.3621549744306467e-06, 2.0942926983212457e-06, 3.601776552903335e-06, 3.571201393271363e-06, 2.6352646499591733e-06, 3.56948272592675e-06, 1.9627113298302322e-06, 2.434172028133289e-06, 8.269033398355377e-06, 2.8264693135995807e-06, 1.0554387207196366e-06, 5.266735208165497e-06, 5.671088016269073e-06, 3.4463848112867806e-06, 1.2086893100711406e-05, 0, 2.09215406099207e-06, 4.787420895326561e-06, 2.7435686379538463e-06, 1.3598306543450737e-06, 6.897528330373355e-06, 1.0617293244669624e-05, 1.2423389339179127e-06, 5.341108468586697e-06, 2.5159925122111328e-06, 1.6354812049935737e-06, 7.834610046305624e-06, 1.1852305633495518e-06, 2.044300847614969e-06, 2.1021069923839593e-06, 4.720594904814508e-06, 2.052903516536373e-06, 3.1435773587876666e-06, 2.130160598868265e-06, 0, 3.050315585959384e-06, 5.81682398227208e-06, 0, 1.4790110308229704e-06, 1.7393812240478005e-06, 1.3426495441571284e-06, 2.532689425806587e-06, 2.924361078368802e-06, 5.665153265939404e-06, 4.090412060478388e-06, 5.6752015827504326e-06, 2.9654745736542747e-06, 1.517093932809086e-06, 3.5215024516172657e-06, 3.5143583103174314e-07, 3.135916432886244e-06, 1.5170938865394856e-06, 3.0564460204734776e-06, 0, 1.8624739586999465e-06, 3.4872579911368566e-06, 4.1924001030824865e-06, 2.3824715150882165e-06, 2.1629902237636e-06, 2.035049300606085e-06, 4.238886082994139e-06, 1.415723213746613e-06, 3.346411669616368e-06, 3.0888970464261663e-06, 3.078969404429929e-06, 1.9353118347445253e-06, 2.7534470459697837e-06, 9.045446542074685e-07, 6.893288958761715e-06, 4.714023819497897e-06, 3.3324479750266033e-06, 2.682155685791515e-06, 4.349663190695333e-06, 8.643150031749049e-07, 2.2382043171567095e-06, 3.4749615239606002e-06, 2.2762474079162614e-06, 3.3959570509735102e-06, 3.4829316153472256e-06, 6.434578720486867e-06, 1.7610783035143842e-06, 2.9801394123852415e-06, 3.331755094268254e-06, 2.287872595779455e-06, 1.5398866524687916e-06, 4.317573968037304e-06, 3.7364013930914078e-06, 1.5038982836461512e-06, 4.18468675582101e-06, 0, 2.9566779946767073e-06, 1.3835779177504034e-06, 1.4912168560930063e-06, 1.170731236739149e-05, 5.135216821850543e-06, 3.1041189320785514e-06, 3.7079704285279347e-06, 3.4154405933019458e-06, 2.3210670598332692e-06, 1.9288123420404612e-06, 2.876720863500277e-06, 4.1759250204567064e-06, 2.568650890288973e-06, 3.7196004868991282e-06, 0, 0, 2.100924910600548e-06, 4.31726282805531e-06, 2.073831566989546e-06, 2.1726389583330896e-06, 2.843916674277632e-06, 1.8326283949402389e-06, 9.253164052920093e-07, 1.6671308848498261e-06, 5.3094052272325384e-06, 5.935354598131742e-06, 2.40566070496039e-06, 3.196709541659838e-06, 1.1414093696187296e-05, 7.392988082187067e-06, 1.3977687959598779e-05, 3.4814707388174384e-06, 4.857681951728661e-06, 4.2509541558748115e-07, 7.332887106712168e-06, 1.4278281306525287e-06, 2.4777463783234716e-06, 4.113621927496776e-06, 3.7079704285279347e-06, 3.7903293258168522e-06, 1.2681097977069702e-06, 1.3792364160138359e-06, 3.3333177351262214e-06, 4.824033493808538e-06, 3.7526541589588604e-06, 2.5497333422556603e-06, 1.2986916223661457e-06, 9.898452315965828e-07, 3.640734609804537e-06, 3.1118248428111854e-06, 1.6204747855052104e-06, 1.8808462930976377e-06, 6.817537721668377e-06, 2.377104568792883e-06, 4.815005888493565e-06, 3.180668313297203e-06, 1.37707955431846e-06, 1.9721155258503334e-06, 6.586214554663617e-06, 0, 3.7655308403633664e-06, 4.890045742577896e-06, 5.63512817036419e-06, 1.1863263345299894e-05, 6.0208860836976585e-06, 5.131555612397337e-06, 3.922247910029905e-06, 2.1811377211661943e-06, 1.625721661380579e-06, 1.6933607938075226e-06, 5.916575640295725e-06, 3.8570102700161104e-06, 1.5967161038115007e-06, 2.1219849112603866e-06, 2.6224132196176876e-06, 2.0126269185119755e-06, 2.447640622374284e-06, 5.201455558849584e-07, 6.390578163090497e-07, 1.8467325593767313e-06, 4.424865396082552e-06, 5.671088016269073e-06, 1.8033817652813678e-06, 4.7556836798656395e-06, 2.6477364570149955e-06, 2.0430604829712774e-06, 3.831805486157604e-06, 2.5539387790174514e-06, 2.074344520215154e-06, 2.6288531902400544e-06, 2.6363357106552874e-06, 3.5813198310047577e-06, 3.0867240610850315e-06, 1.9496714437886567e-06, 2.08128070878861e-06, 6.728460310807267e-06, 1.6393631731575435e-06, 0, 1.0134366157988151e-05, 1.8697924646219836e-06, 2.6144807329241834e-06, 2.9541700230745916e-06, 3.4814707388174384e-06, 3.6897549936168974e-06, 6.647645479834641e-06, 8.654627143259915e-07, 0, 6.428172577151338e-06, 7.048053160400469e-06, 3.5119732612854076e-06, 2.1669639970185224e-06, 8.589366613145109e-07, 1.6858461007246026e-06, 1.4176323374418195e-06, 1.0097793235487945e-05, 1.9032117336168508e-06, 8.235475137413077e-06, 1.1776874145575188e-05, 5.207416873345664e-06, 9.319441552604315e-06, 2.4605221465819757e-06, 6.047568540360974e-06, 2.546823601097236e-06, 1.5656576244342021e-06, 3.386977121925546e-06, 2.6141844808608615e-06, 2.95825052784914e-06, 3.0153717694101596e-06, 2.9434232678869984e-06, 4.797833962159802e-06, 2.661996230041102e-06, 1.4767485611591908e-06, 4.111531029450861e-06, 2.2550937087707325e-06, 3.94993464265405e-06, 1.3787300240818855e-06, 9.231377672767699e-07, 6.150841305947114e-06, 8.106793183437354e-06, 2.5541903441452773e-06, 1.5180464137936595e-06, 1.7661619995980877e-06, 7.755025733371508e-06, 3.208024421977646e-06, 4.27195476478768e-06, 3.2922083144550793e-06, 3.078969592239908e-06, 0, 0, 7.283919540529096e-07, 3.664009208647126e-06, 1.7422975473456595e-06, 4.543392842276679e-06, 5.022013606271158e-06, 4.123766683774007e-06, 2.81142928349814e-06, 2.5158954445040175e-06, 2.7464015930136205e-06, 2.178692839657381e-06, 1.7857025556624185e-06, 2.916442683342882e-06, 6.852541942658125e-06, 3.030278792223175e-06, 1.9192043308004576e-06, 2.0853385315816006e-06, 3.6193863437717564e-06, 6.289867874662126e-07, 7.083180536918024e-06, 1.2528658632181392e-06, 1.876291609482361e-06, 2.7190322665769007e-06, 5.626413547792714e-06, 1.4249720804181732e-06, 2.940294796885222e-06, 1.6593794556891173e-06, 4.405413202424289e-06, 2.993600289523607e-06, 0, 1.7581919413238292e-06, 3.014852991620572e-06, 1.5849972268202672e-05, 0, 4.563359507276876e-06, 8.755619969464653e-06, 3.4978903683557086e-05, 1.404638892695958e-06, 3.4711878495878635e-06, 1.2975734783583868e-06, 9.88297378518235e-07, 2.806678423821965e-06, 1.7641146136636407e-06, 3.485198617831593e-06, 2.153172025767978e-06, 4.984896071474236e-06, 2.407137946755018e-06, 1.8258773002746345e-06, 4.556508579485891e-06, 3.299267260799275e-06, 2.0031532161416674e-06, 1.686881223749935e-06, 2.6924380163072236e-06, 8.852495965717389e-07, 6.817179975372787e-07, 2.231461068972853e-06, 1.0521411279237094e-06, 2.434891149266358e-06, 1.2907792540000154e-06, 1.6709808234162648e-06, 1.0919391682734507e-06, 1.860483272770672e-06, 1.4000546856399637e-06, 1.4767485611591908e-06, 2.6300170560943432e-06, 1.2797565914191218e-05, 5.070625564411638e-06, 2.7769165712183775e-06, 1.0663736326140552e-06, 1.2793167866504428e-06, 4.625191524737922e-06, 3.329590854919091e-05, 1.7338995100216008e-06, 0, 6.210163096779159e-06, 9.471403754330533e-06, 5.414535158017789e-06, 1.699991938212214e-06, 3.7415229442149332e-06, 2.1531823358637622e-06, 0, 2.5644409373324974e-06, 2.44610525107609e-06, 5.507378883758691e-06, 1.4488412715420339e-06, 5.4512532286281555e-05, 1.971182983160287e-06, 0.00012963569138412282, 5.738352319337439e-06, 4.621967526397085e-05, 5.207875645581017e-06, 2.33418524700677e-06, 1.607105725037761e-05, 7.136156341860516e-06, 3.53363472729828e-06, 1.2499329046035311e-06, 7.597200357923277e-07, 1.131247194455039e-06, 3.294517465675001e-06, 7.378649682802989e-06, 0, 2.6149272342345807e-06, 1.4343895523672347e-06, 5.8967237783158225e-06, 3.972650797189359e-06, 4.182615673330546e-06, 4.245240536058265e-06, 1.5781530028502794e-05, 1.1246448964093588e-06, 3.7079704285279347e-06, 2.7466807861315267e-06, 2.149697547190789e-06, 8.039047365689503e-06, 5.902754642879994e-06, 0, 2.469914105300318e-06, 2.098225084351501e-06, 3.863002212689832e-06, 0, 1.6286864271798147e-06, 2.45375515542763e-06, 2.892318836436488e-06, 1.1511613136738819e-06, 7.950371203013894e-06, 4.788316341433018e-06, 2.3176276667370756e-06, 2.8470250004246486e-06, 2.942760563308101e-06, 6.161677705828737e-06, 0, 1.3279925791534484e-05, 1.5635960078904673e-06, 2.3387500608215615e-06, 2.239396158554465e-06, 3.839555894250326e-06, 1.6687307446565112e-06, 1.4832819775624839e-06, 2.841726562840494e-06, 9.592080135542958e-07, 1.3192942969828084e-06, 1.085648130982459e-06, 1.003785449458018e-05, 2.84740017826845e-06, 1.95679551400762e-06, 2.8407898215368806e-06, 3.257663338490307e-06, 2.349438691007965e-06, 1.955447360116968e-06, 1.0272729086157968e-06, 6.0830709833374915e-06, 4.375198293327163e-06, 7.290712105401209e-06, 1.519208630491466e-06, 0, 0, 2.0874923894691166e-06, 3.975971871019743e-06, 1.923115363020747e-06, 5.302013949832155e-06, 5.901005578233218e-06, 2.315632312548111e-06, 4.065142982196784e-06, 4.168624907423479e-06, 0, 2.8117464804875066e-06, 1.7744950959615508e-06, 0, 2.7761395818342198e-06, 7.568900982601457e-06, 1.6777285563165354e-06, 1.4891106022502366e-06, 6.3277526146018195e-06, 1.453897001483397e-06, 4.783241653493887e-06, 4.2379347993958755e-06, 1.0064866094634446e-06, 3.5903916055587537e-06, 2.4161504888471763e-06, 1.2519319536975829e-06, 0, 2.0890313201879498e-06, 3.2253438589575533e-06, 9.543790494937027e-07, 1.3576859130718033e-05, 7.793140034472955e-06, 4.535566400002214e-06, 5.631191864180072e-06, 4.855217078332579e-06, 3.937481257051603e-06, 3.2973619458102297e-06, 3.0879138622535667e-06, 2.102837742995759e-06, 6.512600781715649e-06, 1.557228848367893e-06, 3.6482354038192586e-06, 8.034884279791115e-07, 4.945139584294867e-06, 0, 2.9511081183071126e-06, 1.0051743567228197e-05, 1.92444313363089e-06, 1.977571164188631e-05, 2.2464973057676726e-06, 2.4077337457470748e-06, 9.237047784723253e-07, 3.108445520887541e-06, 1.1047991802159496e-06, 2.5012436963325015e-06, 2.3394148360314106e-06, 1.7669669123764871e-06, 3.60582135714583e-06, 3.906802336101079e-06, 1.569176073939207e-06, 2.7862214357123734e-06, 2.245385847408226e-06, 4.5649388845574325e-06, 3.0164896849139165e-06, 1.1954869048369083e-06, 4.5748484062838664e-06, 2.0173745439766235e-06, 2.1743228167410097e-06, 2.428474965318152e-06, 3.24292058292059e-06, 1.7701720123742627e-06, 5.8219353674405115e-06, 9.306050206602207e-06, 1.79146032119459e-06, 9.037082277083679e-06, 3.248845025483256e-06, 5.4814015899358015e-06, 3.5248984660177433e-06, 4.103284369257986e-06, 1.6792700580467897e-06, 2.569594522610426e-06, 4.531004237475588e-06, 2.059367631596867e-06, 4.739895563609232e-06, 2.307259955504803e-06, 8.106444324348849e-06, 3.2560606243060867e-06, 3.6613364635831905e-06, 1.7268227373301736e-06, 4.455263709460596e-06, 1.0831063977950527e-06, 1.748771930861749e-06, 1.5949249260104092e-06, 1.4803084127565362e-06, 3.648994490190021e-06, 2.9654771060740756e-06, 1.6539723007013931e-06, 2.1105431031213274e-06, 3.2796555863793566e-06, 1.2709730675517272e-06, 2.660950939568237e-06, 3.572558345405707e-06, 2.9802774332401036e-06, 2.0875175111983486e-05, 8.789244413854661e-06, 2.1490135360423534e-06, 8.974739346835901e-07, 3.7990904616496525e-06, 3.6418192395174736e-06, 7.261256159695988e-06, 1.3476864306376023e-06, 5.90503728832816e-06, 7.740119968448772e-06, 2.408114342757181e-06, 3.047971274588524e-06, 1.53102739636381e-05, 4.0613170213628686e-05, 1.8469048600685814e-06, 2.7192280581222502e-06, 1.743306691910485e-06, 1.266954547316925e-05, 4.127079424798497e-06, 4.337142783957362e-06, 1.7831025354031605e-06, 4.2531572064363035e-06, 1.7222599405685232e-06, 4.355773283154849e-06, 4.585239085195049e-07, 1.0724572040575388e-06, 7.426245968952528e-06, 1.8330802340316913e-06, 9.375952429423363e-06, 1.8406443358494186e-06, 1.8438448481572657e-06, 4.35503603907547e-06, 3.885400369835982e-06, 0, 2.182416718526631e-06, 1.0203981078876272e-06, 1.9272588004236162e-06, 2.6218505768011634e-05, 1.8740378812751881e-06, 1.38175614323181e-06, 2.0212747276522966e-06, 2.0617278122119977e-06, 4.669386104865781e-06, 3.2735964333148857e-06, 3.045475023466757e-06, 1.4473357304757048e-06, 1.9178838890216288e-06, 3.7941282468121386e-06, 3.7302407228677565e-06, 0, 4.9393505438566836e-06, 7.652603210411197e-06, 2.918526280700711e-06, 2.687056013497699e-06, 4.89654409041221e-06, 1.7330014903843019e-06, 2.382879368895297e-06, 1.8972656641605214e-06, 1.5037539007935035e-05, 3.691192200964262e-06, 2.5705513589647806e-06, 2.456617455601276e-06, 7.269029332496039e-07, 1.5842134050975166e-06, 5.924686407747557e-07, 2.4235733278335756e-06, 1.3038853177610933e-06, 1.7270571173139855e-06, 4.188611882661377e-06, 2.0805583430470347e-06, 4.0234728821192235e-06, 2.2654286143222543e-06, 3.4850735114451934e-06, 2.369590646079942e-06, 2.6699707650873007e-06, 3.7365962627568935e-06, 2.3192572446877754e-06, 2.8751148979189473e-06, 2.8751148979189473e-06, 1.2836107315483828e-06, 3.177026537666802e-06, 1.1223739016586428e-06, 5.9610045102963165e-06, 3.464137277501801e-06, 4.303015426186096e-06, 2.913672465677903e-06, 1.917225368352918e-06, 2.1047727195849287e-06, 1.4767485611591908e-06, 4.095314225646337e-06, 4.34153808795241e-06, 2.0033181763108376e-06, 8.368771106972237e-06, 1.1544949138868544e-05, 1.9966883344020447e-06, 4.342173049260972e-06, 6.775630853680963e-06, 1.862020898705612e-06, 6.759112293775116e-06, 2.5051298487103623e-06, 3.3899388065392808e-06, 2.2741873483765748e-06, 5.5429320154887775e-06, 1.0291743675849884e-06, 2.9347947640620256e-06, 1.2969993796639063e-06, 1.531182383436109e-06, 2.158786764690916e-06, 8.567340622114776e-06, 1.1893187605640162e-06, 2.3210667766739056e-06, 8.17393389028587e-07, 1.3170986659495495e-06, 3.3976352403193325e-06, 9.394793769888974e-07, 2.9527848994189093e-06, 2.844821389472402e-06, 1.0101381400781587e-05, 1.0666836233348447e-06, 7.061952560079074e-07, 5.396153240236882e-07, 3.374855000587597e-06, 1.6224403765833176e-06, 2.299824836779191e-05, 8.957733161794515e-07, 3.02084227139214e-06, 2.7106327808181447e-06, 3.537514860796029e-06, 5.282966298373974e-06, 4.517653888727148e-06, 5.329531724137591e-06, 1.1519639587541882e-06, 7.086661588484781e-06, 3.8501264013728175e-06, 3.6737240280156913e-06, 5.7083907656968545e-06, 2.736835994800307e-06, 5.953862815410965e-06, 6.850846370608149e-06, 6.537452619881069e-06, 2.6801814927250377e-06, 1.1868977670955935e-06, 3.015888612883136e-06, 1.5899037559250879e-06, 3.1166202004388223e-06, 1.4382520446194976e-06, 4.58499967773065e-06, 1.5964199501398174e-06, 2.175549811896703e-06, 2.106587374297062e-06, 3.498501613910198e-07, 7.520218142360125e-07, 2.268049995810198e-05, 4.785669913771674e-07, 5.71022408605761e-06, 3.800317941730427e-05, 2.5591884906392142e-06, 5.3761724168861e-06, 1.1507473993738775e-06, 8.184782195191816e-07, 7.713633140205824e-07, 1.3455045677236154e-05, 2.125477121292182e-06, 5.402338436428788e-06, 6.075060493175536e-06, 1.7288920680612717e-06, 3.0551320683216265e-06, 5.6666271896531915e-06, 6.897888802948284e-06, 2.925045019684551e-06, 5.411856307547572e-06, 2.184606180983322e-06, 2.1374482182911285e-06, 7.842179702595237e-06, 2.2739572579872533e-06, 2.4693177182717827e-06, 1.598821194475071e-06, 1.4083266244075401e-05, 8.1739875720883e-06, 2.1312953621160958e-06, 3.3057878300338494e-06, 1.4100333159755269e-06, 2.193994820269893e-06, 1.7471329791017867e-06, 2.6168235857324874e-06, 3.3619440871638593e-06, 3.7903293258168522e-06, 1.5037848584651164e-06, 5.747287896533738e-07, 1.463060079495681e-05, 0, 1.492932410787197e-06, 2.4954860842729953e-06, 2.6532937589906064e-06, 2.5982572229499443e-06, 3.6738480351490555e-06, 2.1359463195342382e-06, 3.059645119200605e-06, 8.437894151539991e-07, 1.3845506591300504e-06, 2.7244219824995445e-06, 2.6427479759492073e-06, 1.6721717489983626e-06, 1.1389553179952875e-06, 2.480750000283503e-06, 7.031244112203186e-06, 3.846810650011065e-06, 5.201575808105279e-06, 3.3452484709159853e-06, 4.218907016047365e-06, 2.028114861308631e-06, 5.171098618990992e-06, 3.509301909379991e-06, 6.794681887262867e-06, 3.675922845754021e-06, 4.7397822289032686e-06, 3.8310001330115965e-06, 1.5388046830834689e-06, 5.865661702537432e-06, 3.0310100534042276e-06, 3.1351380052308546e-06, 1.404638892695958e-06, 2.856052857557015e-06, 2.0985283391076336e-06, 2.523348688257879e-06, 1.0805491116736549e-06, 2.405150178327605e-06, 2.0578639530255425e-06, 4.3863226981947284e-06, 2.260074307030831e-06, 2.5012119238218854e-06, 1.4859932812271163e-06, 9.188776963244591e-07, 3.961731202762957e-06, 1.6837891602987442e-06, 3.2524518737335246e-06, 6.923944557091728e-07, 0, 3.96315875301256e-06, 2.226156558329233e-06, 1.5963209685693625e-06, 1.012896480657411e-06, 3.898001441135286e-06, 9.158402540510618e-07, 2.127752773866569e-06, 2.980613725823276e-06, 8.812434123302486e-07, 5.027504464534309e-06, 6.933072999528925e-07, 4.308855427859696e-06, 2.295886303780593e-06, 2.0773380416465545e-06, 2.5586315312101536e-06, 5.301819583673225e-07, 2.4494252131676927e-06, 1.8027747659076015e-06, 1.3817560168059486e-06, 1.2210450820662463e-06, 1.6813987048422477e-06, 1.8764593139951448e-06, 8.197212032025149e-07, 3.955688827379561e-06, 9.362899903434431e-07, 2.036830616407238e-06, 1.4839433743086773e-06, 4.069450407922544e-06, 2.6352646499591733e-06, 9.72948321985395e-07, 1.3733700761811259e-06, 3.5458381079659723e-06, 1.7888568822843937e-06, 1.1348789704291893e-06, 1.484967974046644e-06, 3.3497887899008407e-06, 2.8760067758882735e-06, 1.5890739655205175e-06, 3.0209882122842045e-06, 1.8318187396658493e-06, 4.2703242404620195e-06, 4.486306355338887e-06, 3.7035371732255723e-06, 4.139291336982349e-06, 8.2213550401742e-06, 2.0930633348866488e-06, 4.499283437790407e-06, 4.129363768005411e-06, 4.3812970080152805e-06, 3.366673214811206e-06, 1.9567519182624387e-06, 2.651427253417638e-06, 3.6558894074022117e-06, 3.1528169576999293e-06, 3.3341429285943192e-06, 1.5636308681045712e-06, 4.560750194729986e-06, 6.7409084740832025e-06, 5.15131580260797e-06, 2.9118844564064585e-06, 2.132189847694255e-06, 1.8913185015770392e-06, 1.0000708818092023e-05, 1.5812256659319914e-06, 0, 1.8483901622900523e-06, 1.9931121562060463e-05, 2.229862949320546e-06, 2.0977060978775857e-06, 1.9779843254865444e-06, 6.732035406568479e-06, 9.12336805774724e-07, 0, 1.754641678927759e-06, 1.8320669236484026e-06, 7.825989898806694e-07, 3.346411669616368e-06, 9.895868162061806e-07, 1.871769077541879e-06, 7.90200736637325e-06, 2.2699291289977073e-06, 1.9626440428911276e-05, 1.9819016871752626e-05, 3.33191956164751e-06, 1.7842996251496487e-06, 2.3723593991967706e-06, 7.381114491240321e-07, 7.917330532886013e-06, 2.895524528683439e-06, 8.045763568633896e-06, 2.863499334839414e-06, 9.835105679781774e-06, 3.3040338295821503e-06, 1.4346192436248041e-06, 1.4342234975586356e-05, 2.6224132196176876e-06, 4.171977681825619e-06, 1.9971840339371934e-06, 0, 1.6846155656057263e-06, 5.12496599432064e-06, 3.7931897863699865e-06, 2.446136435481588e-06, 4.757849662289373e-06, 2.407822160797589e-06, 2.655089224459038e-06, 4.8193323311015925e-06, 4.5513972792649436e-06, 5.777094850841831e-06, 6.048825747789038e-06, 2.986304995688262e-06, 3.225429146213327e-06, 1.694970962834353e-06, 1.553967458614904e-06, 1.4417989275917223e-06, 3.067392549149029e-07, 1.0934475305239668e-06, 1.9669270258539594e-06, 1.3488490743654465e-06, 8.880453968634546e-06, 3.2197152681299876e-06, 0, 2.6288407628398577e-06, 1.7596828045546422e-06, 2.3478783094919677e-06, 9.693989420830117e-07, 9.447132806287777e-07, 4.588342234502536e-06, 2.9410225145140508e-06, 2.0464897132422477e-06, 4.009374496774424e-06, 4.870383146718152e-07, 4.8235502657700006e-06, 4.66111068948003e-06, 2.119892281793015e-06, 2.63115415305454e-06, 3.765530725519056e-06, 8.591234369607981e-07, 1.5387082413528897e-06, 1.0028835752286796e-06, 1.1598615985957449e-06, 1.357368234778073e-06, 5.473091122322304e-07, 5.669005422341171e-07, 2.487006256204292e-06, 1.4505174985891987e-06, 1.075182941514171e-06, 6.047568540360974e-06, 6.360095029465938e-06, 7.137950538185001e-06, 1.3292298502362597e-06, 4.610784620107839e-06, 2.169242320758348e-06, 3.5658351161889567e-06, 4.984255671356401e-06, 3.1084032386030238e-06, 3.95117566222198e-06, 0, 1.633069453069223e-06, 1.6854289601141737e-06, 0, 0, 1.3909958431693536e-06, 1.1118298499059725e-06, 1.1887407120308608e-06, 1.0773429746826445e-06, 4.465999895119555e-06, 6.903565968105612e-06, 2.9789587477838178e-06, 6.303517624751892e-06, 7.413482355862835e-06, 3.265989929125644e-06, 2.2203796008661455e-06, 1.472580274961317e-06, 2.728219591024378e-06, 2.1167951833126816e-06, 4.3059986011162445e-06, 4.138426974384584e-06, 5.202862791979474e-06, 4.292995422549015e-06, 3.578554857411383e-06, 9.636682295850199e-07, 1.6823164182907326e-06, 1.5523067060253275e-06, 8.870286504771987e-07, 1.8214587023885064e-06, 1.5566798708521687e-05, 7.951437203696959e-07, 1.4379786380405637e-05, 2.798035799790286e-06, 3.701579979173678e-06, 3.403924394098956e-06, 6.854231558450379e-06, 2.2124178308857397e-06, 4.941600177596602e-06, 2.9226687494000617e-06, 1.7882159399955553e-06, 9.811614845422348e-07, 6.255713455213362e-06, 1.5767362249410428e-06, 4.551230684390048e-06, 5.401604210928941e-06, 3.846520931153187e-06, 2.2240108830542135e-06, 1.1157801548631054e-05, 1.359340197171998e-06, 3.19751477251812e-06, 1.4305066525742452e-06, 1.3256316182902752e-06, 9.314801505355086e-06, 7.325643262415831e-06, 2.2118571074979998e-06, 3.2380600167144822e-06, 4.704086060723834e-06, 2.428442006322161e-06, 1.1823260091888317e-06, 3.6632835516258636e-06, 0, 5.462455293468804e-06, 1.5194688187208676e-06, 1.3697986734803811e-05, 3.835399912025119e-06, 7.820845214332859e-06, 1.3876255684327696e-05, 2.2862123903819925e-06, 2.8926847276952853e-06, 0, 1.9752146385131183e-06, 2.4866124321182088e-06, 3.880086182827741e-06, 5.29222349985105e-06, 2.378102000705865e-06, 2.12869511335155e-06, 3.526378170434256e-06, 5.38783932710766e-06, 1.1644515391744825e-06, 1.9214336144092576e-06, 3.3439657375981084e-06, 2.3701222497787326e-06, 4.2335129383480945e-06, 8.098224151981346e-07, 2.865222504412681e-06, 1.9884847307301856e-06, 1.1552437166700566e-06, 2.9629212769942326e-06, 7.524872795494146e-07, 1.7787662941140112e-06, 3.635128726504656e-06, 3.6558894074022117e-06, 2.6322477475636298e-06, 0, 4.663233740733368e-06, 3.4209126962776176e-06, 1.433084947903739e-06, 3.874495306901447e-06, 3.052985769098069e-06, 3.617573579803152e-06, 3.3928166248451882e-06, 4.394639374367129e-06, 2.4721332920082714e-06, 6.297783495715675e-06, 3.2174094823158287e-06, 1.6323899319945565e-06, 6.174975913427173e-07, 1.5663708485880646e-06, 1.8723750947079475e-06, 2.705724497099778e-06, 1.4349940912231986e-06, 6.593526932726702e-07, 3.9983095216033064e-06, 2.9667189758146393e-06, 1.1599592715088788e-06, 2.600084316321578e-06, 2.1625903255745753e-06, 1.2000150821452179e-06, 1.498773629460135e-06, 2.482757466336323e-06, 3.970630452862299e-07, 3.0868375036698887e-06, 2.02117467781283e-06, 2.3172419710703447e-06, 1.678942411430228e-06, 1.4913306068612393e-06, 4.86402506566441e-06, 1.8231903424731641e-06, 8.373623158224577e-07, 3.4021937587180957e-06, 4.959350032748222e-06, 5.2483931575171e-06, 2.1837378915972686e-06, 2.9649366718909308e-06, 1.3783746065846556e-05, 2.4735278546523498e-06, 2.539283322938851e-06, 1.7676839892539323e-06, 4.142647683037198e-06, 1.9276401091092673e-06, 0, 2.6646069221469328e-05, 4.444981093712131e-06, 4.885831724380842e-06, 2.5403833989991694e-06, 2.8348421882311825e-06, 1.7307431676759446e-06, 8.934117481760949e-06, 1.3427618726473382e-06, 1.07086090318777e-05, 1.6162482048380978e-05, 2.288030856517241e-06, 3.1157963874060646e-06, 3.7328834084623096e-06, 2.039669968681272e-06, 2.688583737384352e-06, 2.871917874707845e-06, 2.788870477404833e-06, 1.9272588004236162e-06, 3.3999839107682334e-06, 1.5859095993700855e-06, 1.3102032846692323e-06, 1.3839202663594573e-06, 2.2709341238954874e-06, 5.260375881509958e-07, 1.672807110500799e-06, 1.9768936316211415e-06, 1.2558858697417808e-06, 0, 1.731290852685992e-06, 3.202274133030102e-06, 3.323430518401753e-06, 4.184356924684853e-06, 2.030967991566613e-05, 1.3104065786433186e-05, 1.7578484135485045e-06, 1.7459054722600178e-06, 3.202648165428553e-06, 2.0048793717718557e-06, 1.9971840339371934e-06, 2.5539387790174514e-06, 1.5164018049743288e-06, 2.387202072488029e-06, 4.386322430639261e-06, 5.368026960349509e-07, 2.113484824225016e-06, 3.2605147312348404e-06, 1.2943977986044187e-06, 3.2014750836649146e-06, 7.408019677119176e-07, 1.4044616754058887e-06, 4.313468330531047e-06, 1.5815037106353252e-06, 1.1630806733885059e-06, 5.437520349479515e-06, 1.5237090186669438e-06, 4.6685821936473565e-06, 2.1737572623373466e-06, 2.7351564615930853e-06, 5.863663135291941e-06, 4.333353424383096e-06, 1.347901251349223e-06, 1.6849728660853631e-06, 1.8735503470723346e-06, 5.924015717051149e-06, 1.6921926650258538e-06, 3.826760274546709e-06, 1.6531612512848718e-06, 4.208882162248044e-06, 2.7190231376761963e-06, 3.8928157630010824e-06, 0, 2.458313270513341e-06, 4.416267252159845e-06, 6.020200172818711e-06, 7.246387620779721e-06, 5.05144349988977e-06, 2.1583266271483293e-05, 1.162625294273477e-05, 9.84103508684523e-06, 6.16161446972587e-06, 1.4483534300847842e-06, 1.5642360622476243e-06, 1.8446844328201856e-06, 1.8666532849974662e-06, 3.2290098013328664e-06, 7.42041098202656e-06, 2.735191330995439e-06, 7.225962649980647e-07, 6.3293186253775995e-06, 2.837323534078969e-06, 4.8302260731350916e-06, 1.42774121329166e-05, 4.771301892897747e-06, 2.3739443045282983e-06, 7.210086612086593e-06, 4.514432771048835e-06, 2.2395286624925673e-06, 1.3481085409853095e-06, 8.639732001250626e-06, 2.3860337351416105e-06, 1.1616147347136573e-06, 1.8396676339608337e-06, 3.943082627114386e-06, 2.312360211124449e-05, 3.481208588271286e-06, 1.078286962805754e-06, 1.1757326517631619e-06, 1.109051711911471e-06, 3.3942834173125007e-06, 2.3803876639945687e-06, 1.0766006463497782e-06, 1.5886441399295725e-06, 3.2922083144550793e-06, 3.6979773140129423e-06, 4.763192957485933e-06, 1.2023326838830878e-05, 2.9913095943741513e-06, 8.928519768167081e-07, 1.1557196971801404e-06, 0, 4.322566782851525e-06, 7.153845337716639e-06, 7.161956466731862e-06, 2.1670617461582286e-06, 3.660079314623248e-06, 4.545461997926265e-06, 3.7356242946951495e-06, 6.187866195956278e-06, 5.1641661414416366e-06, 3.7584844408423865e-06, 1.3721507662531713e-06, 1.7006380583453443e-06, 5.165107597966674e-07, 2.859334758143094e-06, 3.261633943065897e-06, 2.063735739894348e-06, 5.818614997808518e-07, 0, 2.115494122484427e-06, 3.351968585928633e-06, 0, 2.1518487704553962e-06, 1.0203074254527793e-06, 2.49272054556423e-06, 5.066259639126647e-06, 1.6342398819326657e-06, 1.3654396073004776e-06, 7.290732099140382e-07, 2.5570611240151336e-06, 5.620037124617423e-06, 2.5123333504685197e-07, 2.9469333849069644e-06, 3.04606442596116e-06, 6.918742400843681e-06, 2.565822071996494e-06, 4.237180745211639e-06, 1.961063820083113e-06, 3.488201561313434e-06, 3.2231625213304036e-06, 2.101150181033755e-06, 2.2283264929788023e-06, 2.2555932280009987e-06, 4.255546733704533e-06, 7.700759368100008e-06, 1.4226393645939317e-06, 1.34944513584794e-06, 2.0369437107636374e-06, 3.126654317606345e-06, 2.2312929745948296e-06, 2.3906479588535455e-06, 2.5982572229499443e-06, 2.8246959636739033e-06, 1.9484487685470403e-06, 3.6540343447141163e-06, 2.2197998326722762e-06, 5.633555826338398e-06, 1.903382773087985e-06, 0, 5.5486684135703145e-06, 1.4588346697183409e-06, 1.4859929639796826e-06, 3.3675897636940173e-06, 1.5849148284286536e-06, 4.876306476828733e-06, 5.42783253607835e-06, 1.7971989679185393e-06, 5.805932304714234e-06, 5.486198807854313e-06, 4.104813618093119e-06, 5.9549201023370825e-06, 2.771698944544317e-06, 5.910159389550492e-06, 2.9143512850823212e-06, 5.797439669595501e-06, 1.748615691464776e-06, 5.528240107939892e-06, 2.0752631113046434e-06, 1.0169446375094961e-05, 8.535676022367931e-07, 4.569720296234025e-06, 2.189788075950388e-06, 3.4196831341054167e-06, 2.8931107830946853e-06, 3.814194581529472e-06, 1.1535853883609754e-06, 1.1923057285460546e-06, 2.128855283740452e-06, 1.3396835948320005e-06, 3.7903293258168522e-06, 2.6168235857324874e-06, 1.0386393599624688e-06, 1.3254298901989476e-05, 4.5489782153361775e-06, 4.040372850225179e-06, 2.5496683713436477e-06, 9.528734329454935e-07, 3.346411669616368e-06, 2.901423198915351e-06, 8.741155607147655e-07, 4.724306178305931e-06, 4.305558631188196e-06, 4.581391481667339e-06, 0, 4.530989808121848e-06, 9.627399583260147e-07, 1.4518917251813489e-06, 8.412388874391871e-07, 1.465389917447506e-06, 4.584210924175302e-06, 9.394793769888974e-07, 3.4214577797774924e-06, 4.117222530290564e-07, 3.389697720134574e-06, 3.912880224094224e-07, 3.779009193801209e-06, 1.4030308346816248e-06, 1.7802347775242443e-06, 6.797044822460167e-07, 0, 0, 5.128881378985449e-06, 2.145076213101576e-06, 0, 7.304862003033135e-06, 0, 1.546102031878373e-06, 2.638295953333604e-06, 7.179638851911384e-06, 3.95997795082876e-06, 2.0530129089907875e-06, 5.646892293960424e-06, 1.2226579504715685e-06, 1.5897840380905247e-06, 2.5860514098491975e-06, 7.908266618394013e-06, 6.172934758166708e-06, 3.870096240243154e-06, 4.756452541485589e-06, 4.340998543141447e-06, 1.4094417858792609e-05, 2.979624106184326e-06, 2.7526739788955283e-06, 2.9556327891756034e-06, 6.550833810596709e-06, 2.396457524256067e-06, 2.577912159771838e-06, 2.001188302779876e-06, 1.7866293555633434e-06, 4.434855707512938e-06, 4.300131345295927e-06, 1.127555130743324e-06, 2.2788327953504736e-06, 2.4461031621827516e-06, 0, 3.303999316315094e-07, 1.9616027837257916e-06, 1.7121189158830263e-06, 4.936792359972179e-06, 1.1511201182927661e-06, 2.682461276919905e-06, 3.6754821063203033e-06, 3.116870877930959e-06, 3.276492899906867e-06, 3.1618638383586287e-06, 1.3457935560327088e-06, 5.199274025333852e-06, 1.2106471417803758e-06, 2.3685663352979807e-06, 1.3464778292544935e-06, 2.946825338502797e-06, 3.4696075063916245e-06, 6.31479614556781e-06, 2.4589529713906544e-06, 4.407353250136114e-06, 2.3083476258802146e-05, 3.3589635391805065e-06, 2.140216790371388e-06, 4.646897650816854e-06, 0, 5.369411952591235e-06, 8.966656322756356e-06, 3.3848954117864607e-06, 3.1118969729965358e-06, 2.72536718989865e-06, 3.83445112627715e-06, 1.358770389448388e-06, 1.9903506362450103e-06, 5.6445239405909435e-06, 1.3488492183496022e-06, 1.0357524622439796e-06, 8.308024258239456e-06, 5.332144807135124e-06, 3.56722106324563e-06, 3.544059154003406e-06, 9.708841357172775e-06, 4.613283674022167e-06, 3.740709128191958e-06, 8.223793740114973e-06, 1.067420171807456e-06, 3.3837570203247283e-06, 5.570396983952255e-06, 2.536410957800492e-06, 4.315763273402729e-06, 1.5897623162581376e-06, 2.9774842672125617e-06, 4.304080414698837e-06, 2.5176322560834e-06, 1.8367907740016269e-06, 3.162488545921704e-06, 1.9621332481920115e-06, 1.4305092266751277e-06, 2.37453170659666e-06, 1.8170008665739346e-06, 1.2207929715964398e-06, 0, 2.1956383151820646e-06, 1.0772471001575939e-06, 1.8182568701563257e-06, 7.75425944881152e-07, 1.7511235798728094e-06, 1.2034782788723812e-06, 4.130126367932555e-06, 1.1266992698003385e-06, 8.050067754632953e-06, 4.294701742727358e-06, 3.379344666067678e-06, 4.991416108860322e-07, 2.8612707554872857e-06, 5.426828268368562e-06, 4.386817246640159e-06, 1.5112552389891946e-06, 1.7140595867812297e-06, 7.064126523112699e-06, 1.1226668211557398e-06, 1.5942191698325157e-06, 5.698141612867454e-06, 1.256179650252608e-06, 2.950972708341428e-06, 1.5028415549060726e-06, 1.3427618726473382e-06, 1.7630558107355879e-06, 1.498773629460135e-06, 3.2605147312348404e-06, 5.528240107939892e-06, 4.113621927496776e-06, 4.921301522174845e-06, 3.501352204270143e-06, 0, 2.2366743921508692e-06, 2.8973844956120687e-06, 3.389358325936464e-06, 8.23405287086005e-06, 4.326631783124467e-06, 1.862615688300101e-06, 3.5961443929810097e-06, 4.012855703863253e-06, 4.1470572060667255e-06, 6.737924278318333e-06, 4.551292407762558e-06, 1.5082384754415737e-06, 6.312405897398901e-06, 7.866381557793518e-06, 3.706190649831588e-06, 1.3348396155217786e-06, 6.07240506765097e-06, 1.3678136250590319e-06, 4.0512456952349785e-06, 2.6197351195260647e-06, 1.2182790195903132e-06, 2.057566574610584e-06, 7.676112767283663e-07, 2.733974133416154e-06, 2.166348192926184e-06, 1.8559493434421284e-06, 2.2654287525081313e-06, 3.204593017277094e-06, 4.4715747476013185e-06, 1.4349938286293182e-06, 1.4205321853788492e-06, 3.6507505693975596e-06, 3.1455050462042087e-06, 4.6778571136148175e-06, 1.3567675356490197e-06, 9.700892178730394e-07, 2.6486611582030843e-06, 2.2788327953504736e-06, 1.922397235167023e-06, 4.354700074904859e-06, 6.681946629534692e-06, 1.9598871091963077e-06, 1.445887668915461e-06, 3.0808788968483997e-06, 6.013837351008899e-06, 1.301081781065777e-06, 1.805598403614473e-06, 1.8222708368583736e-06, 1.0972086343444453e-05, 3.833938994160098e-06, 1.3479183118667261e-06, 2.7809666060192276e-06, 1.5977182818170964e-06, 3.2175686977140163e-06, 1.5242676137534598e-06, 2.4691251610142364e-06, 3.0556870401515175e-06, 1.9472514221736812e-05, 0, 2.299612035849008e-06, 2.4914314556712578e-06, 3.9128804627708175e-07, 1.8904991765054298e-06, 7.075045437138505e-06, 9.158403099151891e-07, 2.4735278546523498e-06, 7.55635482031054e-06, 0, 5.023072679181338e-06, 1.5358897004272145e-06, 7.212518558166918e-07, 3.3452455121578443e-06, 0.00011289980922291698, 6.486711161868964e-06, 2.15221898202335e-06, 1.364763928863363e-05, 4.1879767049354545e-06, 5.010700692306272e-06, 5.982416080157773e-06, 5.091001315430109e-06, 3.0344112941245187e-06, 2.865222504412681e-06, 1.5956312866138647e-06, 4.625519719363517e-06, 1.942811909251047e-06, 2.9793289585760394e-06, 2.188569437286121e-06, 2.030814902122065e-06, 2.3839340808756043e-06, 1.8219326813332873e-06, 4.8607313920617845e-06, 4.423887767811549e-06, 3.5930296198831976e-06, 2.4491567892741617e-06, 4.282348281021713e-06, 9.5486363931959e-06, 1.0734623415078501e-05, 0, 4.965825979603168e-06, 0, 3.182282962725453e-06, 1.4767485611591908e-06, 2.9802774332401036e-06, 3.546715750551255e-06, 2.2960903565542637e-06, 4.355785560358441e-06, 7.332887106712168e-06, 8.745596098136852e-06, 5.688091972355084e-06, 7.138965741695534e-06, 4.705628032922657e-06, 1.823441834266584e-06, 2.7307501894863657e-06, 6.66085672804711e-06, 5.050245107773434e-06, 2.8302051977622896e-06, 5.02833682516697e-06, 5.191076939752097e-06, 5.26867066486712e-06, 2.236912614995536e-06, 2.7792195162628746e-06, 2.91005782695958e-06, 5.280560615558203e-06, 1.2000151187442831e-06, 1.6641909463004535e-06, 3.066066877766043e-06, 4.06608571732039e-06, 3.592915819506518e-06, 2.7591910047343947e-06, 4.019310227547993e-06, 6.925457137985267e-06, 4.294458895439065e-06, 1.0367311959298562e-06, 1.0014874812753976e-05, 4.40137605805057e-06, 1.9634878679741557e-06, 4.457492104358002e-06, 0, 0, 1.5870031503932285e-06, 2.4855692609571627e-06, 3.2615065250346643e-06, 9.143965513489963e-07, 5.39720704351008e-06, 5.377743518645875e-05, 1.1501842390795969e-06, 2.108693302315612e-06, 2.2281566979739664e-06, 1.581845413252733e-05, 6.048056967285077e-05, 4.5256465889038955e-06, 1.1157822477098895e-06, 1.5570087307713787e-06, 1.5798427284146231e-06, 2.3951198543415135e-06, 1.7701720123742627e-06, 2.246466645265966e-06, 1.0792306918641462e-06, 2.2121067257606154e-06, 1.5870017951447953e-06, 1.748615691464776e-06, 5.226121761106958e-06, 4.810493477594964e-06, 1.4298579523501723e-06, 6.1244584496145435e-06, 5.2118953029750435e-06, 2.7720994513810843e-06, 3.967133244472507e-06, 1.643403285981707e-06, 2.7962294182595506e-06, 1.5012883709506953e-06, 1.0187886046027843e-06, 2.497036077188254e-06, 1.9688423263092167e-06, 6.576326433166794e-06, 4.088910000612076e-06, 5.7520129081597445e-06, 3.885729567441115e-06, 1.4700026095121008e-06, 3.1427994257550563e-06, 4.516191901240147e-06, 2.966983023828944e-06, 2.870951042173737e-06, 1.1043891365387322e-06, 2.351166537558941e-06, 1.092234737979348e-05, 4.872042261106231e-06, 3.498501400509749e-07, 1.3096961347151386e-06, 2.7574222736296875e-06, 1.6998623496329593e-06, 3.0361996325376466e-05, 3.7863205097835074e-06, 1.46833373868589e-06, 1.4755257153321793e-06, 3.5062137517433205e-06, 2.119892281793015e-06, 1.588623233116011e-06, 2.661996230041102e-06, 1.4488412715420339e-06, 3.7447979675467624e-06, 3.904892483759022e-06, 1.4646447670043197e-06, 3.2563711360656884e-06, 8.38042136981703e-06, 1.9086652095445644e-05, 1.2134233205342542e-05, 1.9342659085223557e-06, 4.578555619893701e-06, 1.6973230194492125e-06, 2.9415289019949307e-06, 2.0258380462245936e-06, 2.842018913612964e-06, 2.6168235857324874e-06, 1.2844968142450939e-06, 1.587276208155606e-06, 4.413685710836014e-06, 4.556508579485891e-06, 4.589127639955525e-06, 8.199391494072467e-06, 2.252392294317522e-06, 0, 3.3619866908861667e-06, 1.362147206101985e-06, 8.068925660549157e-06, 0, 1.7583241265178683e-06, 2.3564286381836896e-06, 2.1665410630534727e-06, 1.6710566580390254e-06, 9.569463680221738e-07, 1.0237545025553463e-06, 9.662708214092424e-07, 6.292129616335187e-06, 4.117261896830163e-06, 2.897642273201772e-06, 2.034922161072033e-06, 1.232857660695137e-06, 7.746826400241963e-07, 3.0200247550359654e-06, 4.738723027378131e-06, 4.184629578832099e-06, 0, 3.50739376899493e-06, 0, 2.6181912093040653e-06, 2.886520371415875e-06, 1.7486622231588606e-06, 3.906802336101079e-06, 2.53481990209181e-06, 4.970120512941056e-07, 1.6698826266315865e-06, 4.709477706171256e-06, 5.351907739955394e-06, 5.768065418316969e-06, 4.446949619828468e-06, 1.3430941218495623e-06, 2.1093140754749316e-06, 1.2340237779116613e-06, 1.7795933694601602e-06, 3.677074409880432e-06, 2.8441764175429715e-06, 2.6695303004112995e-06, 2.408114342757181e-06, 4.063527743661339e-06, 3.547765810832343e-06, 2.2259577701006392e-06, 1.2302874794325888e-05, 1.251943943050132e-06, 5.0141603678607685e-06, 4.991348289792946e-06, 6.121810900801249e-06, 2.9983276750271375e-06, 2.8452186601897526e-05, 9.445983251245146e-07, 2.0738854140545087e-06, 5.96985737319605e-06, 1.941852387647666e-06, 2.9194200500605544e-06, 1.7625972562250637e-06, 4.272579875124887e-06, 1.4470719652617956e-05, 0.00024585348944594545, 3.5699161531624033e-06, 2.62258326378693e-06, 2.1302530549285577e-06, 1.322533481389658e-06, 2.7526739788955283e-06, 5.823788879711562e-06, 3.970191501128833e-06, 6.707937817381023e-06, 4.527131381362187e-06, 2.719901145088049e-06, 9.239545985905388e-06, 3.657417630302864e-06, 3.3402687119033116e-06, 3.5780277506235394e-06, 3.0713950166678195e-06, 1.1703216005992982e-05, 1.2324157621896527e-05, 7.877312958144221e-06, 2.1247361110659084e-05, 1.4797351933171711e-06, 3.927151359713192e-06, 1.4372401059048028e-06, 5.917861657304582e-06, 2.174190192151567e-06, 1.283446360219671e-05, 2.7769165712183775e-06, 1.5849148284286536e-06, 1.4643025010828248e-05, 3.1806682162905194e-06, 1.98609230324545e-06, 1.94130064419545e-06, 1.231472626960086e-06, 5.014954575781096e-06, 2.5673800067451013e-06, 7.184428898940041e-07, 4.805748213140165e-06, 0, 8.43284756296942e-06, 3.881368006170875e-06, 4.355394411715354e-06, 1.2353822165897991e-06, 2.1071411082079624e-06, 3.4006451379169206e-06, 2.116430903758578e-06, 3.2924766671738654e-06, 2.9749697616906217e-06, 0, 9.204688435987024e-07, 4.190329228810497e-06, 1.8535055915013772e-06, 1.785531335811405e-06, 2.486612128763081e-06, 2.6141844808608615e-06, 1.447335752546735e-06, 2.267181281585795e-06, 7.27726893171702e-07, 4.689746769619131e-06, 2.6897389847109336e-06, 5.214676198084189e-06, 4.126613780361347e-06, 2.693682282468477e-06, 1.4693674579155199e-06, 0, 2.5729630182832818e-06, 3.3339232400280465e-06, 1.9883478563545604e-06, 1.133197496571653e-06, 2.089755804484949e-06, 2.0893899915969347e-06, 0, 3.479667618698845e-06, 9.715438992323234e-06, 1.6345210671875554e-06, 1.4525793100177167e-05, 3.0426525824932364e-06, 4.454341516960777e-06, 3.469054381695697e-06, 3.3123550783323725e-06, 1.7082405163662446e-06, 2.00265777580324e-06, 2.0831423586243642e-06, 1.2618123075335556e-06, 1.5881003131405269e-06, 1.3027796680930654e-06, 2.4735278546523498e-06, 6.742223555146895e-06, 3.204900217380445e-06, 3.145009201173775e-06, 2.2558680038592463e-06, 3.805081144352369e-06, 3.1447828481826677e-06, 2.5657454716789665e-06, 4.683630929818161e-06, 8.696705531192364e-06, 1.6166544358912433e-05, 1.370042486414675e-05, 3.403924394098956e-06, 7.301836939922133e-05, 4.046701547985515e-06, 1.2478237461861434e-06, 0, 4.362607777263528e-06, 2.4573951022123963e-06, 1.5561124351279288e-06, 8.048830665525125e-06, 7.47096748548607e-06, 7.1301020110882646e-06, 4.228388410189067e-06, 7.96018375384935e-06, 5.870247864891646e-06, 1.126953965769079e-05, 1.8929735741848573e-06, 3.6550723775044395e-05, 1.6279671217277933e-06, 4.170369990265333e-06, 3.795518545920691e-06, 5.97292266520294e-06, 3.1089381613665593e-06, 3.7526322987693096e-06, 3.0193448498620733e-06, 2.8904564353527546e-06, 1.6726467154880238e-06, 3.0092802153242388e-06, 5.639246783288876e-06, 4.952006250622625e-06, 4.143742046543536e-06, 4.185406959226426e-06, 2.8706981781012457e-06, 0, 1.9356014368028422e-06, 1.2620282586569085e-06, 4.458770462456302e-07, 1.4090603122936517e-06, 3.998258122626031e-06, 2.471430727743984e-06, 2.344517560707692e-06, 4.0210000290974335e-06, 5.395483305024605e-06, 3.043967991011336e-06, 7.912946421863036e-07, 1.6967857694895627e-06, 1.4720329445080103e-06, 5.677251231314955e-06, 1.7634922176351995e-06, 1.4633046115920721e-06, 1.5253051924389916e-06, 2.184207549031663e-06, 5.535018214345859e-07, 2.82528391205767e-06, 2.1805997562588553e-06, 1.6046481957006598e-06, 2.2762474079162614e-06, 7.252562627401865e-07, 2.267181281585795e-06, 7.956503629831557e-06, 4.465956649351527e-06, 4.919721073836549e-06, 5.5832874620327395e-06, 3.82789447219468e-06, 9.467999565492735e-05, 5.77099407421164e-06, 1.960967647694757e-06, 1.151074509743342e-06, 4.98372593071495e-06, 1.8170023074012543e-06, 1.4405380642878714e-06, 4.084011889244901e-06, 4.834145846055764e-06, 3.8037976097028955e-06, 1.5859095993700855e-06, 1.56457735036784e-06, 1.6778015757399705e-06, 1.4531014103584448e-06, 9.952707231798594e-07, 3.462957396699405e-06, 8.301591761160763e-07, 2.1954077688056317e-06, 1.6609465481855702e-06, 7.797380003037223e-06, 1.166111513433812e-05, 4.415934375861925e-06, 1.6724982209676885e-06, 2.0817671871352286e-06, 6.343025147252277e-06, 3.443957503529498e-06, 2.4491853297028415e-06, 3.178390199987125e-06, 6.523904828739389e-06, 1.7471307677539414e-06, 3.72289173242936e-06, 8.63739452973281e-07, 1.6434059925659395e-06, 5.301034510701003e-06, 3.6541875367124413e-06, 1.4330144932677923e-06, 1.2586312501670914e-06, 0, 7.043632013590067e-06, 6.955812610197714e-06, 5.19160235726526e-06, 6.728138754085339e-06, 2.27751882671737e-06, 2.5982572229499443e-06, 2.0422312922903503e-06, 6.8666231874350784e-06, 6.047568540360974e-06, 4.245135339006108e-06, 2.595158855302674e-06, 2.422523723848389e-06, 0, 1.327344458680112e-06, 0, 5.589314515443855e-06, 2.5767037929311167e-06, 2.4630638913451115e-06, 4.86402506566441e-06, 4.4010446397437675e-06, 5.365832024926708e-06, 5.071011967617935e-06, 1.9553065329659108e-06, 3.8220880653639044e-06, 5.2851280622813995e-06, 9.823602799019942e-06, 2.2342025890728623e-06, 3.1235202365660848e-06, 1.9391044043687745e-06, 3.2161976019625605e-06, 2.8984765033258954e-06, 2.7203322902673093e-06, 2.7036516924675738e-06, 2.4632285705606067e-06, 5.19160235726526e-06, 4.847744945096594e-06, 1.221814302410941e-06, 1.9272588004236162e-06, 1.4164251583046726e-06, 4.991416108860322e-07, 1.8844339043265868e-06, 2.3504182060417966e-06, 2.0739134845525543e-06, 1.8467753091767814e-06, 2.9587971504948133e-06, 3.96315875301256e-06, 8.690250122361264e-07, 6.695130397983632e-07, 1.4903951156829342e-06, 1.2407198937875247e-06, 1.5781528785097173e-06, 1.0439025598203554e-06, 1.1016032162682494e-06, 5.814965060648133e-07, 1.7233590493546028e-06, 2.251357979969814e-06, 1.686881223749935e-06, 7.135708915133143e-06, 4.690966918245464e-06, 6.891424931608888e-06, 9.792676436605374e-06, 1.4489475033253903e-05, 2.8432377847306663e-06, 0, 4.5732195725761035e-06, 4.1766399581378747e-07, 6.132532095778812e-07, 1.2859392878292266e-06, 3.2537298188037147e-06, 2.933634345542614e-06, 3.0887032671268587e-06, 1.496137565651356e-06, 8.449185032608634e-07, 5.490318416013736e-07, 2.2216102661480468e-06, 2.7226912614576106e-06, 1.5629201547424557e-06, 0, 1.9722654184435283e-06, 3.570810037871977e-06, 3.1132747906149956e-06, 2.412603311144707e-06, 6.732032532099577e-06, 4.454190383912329e-06, 2.466955353902856e-06, 2.4410152190365247e-06, 5.63035279590389e-06, 2.6776036781532384e-06, 1.1880680087807787e-05, 2.2622229684828583e-06, 6.006639045365603e-06, 2.622446691692879e-06, 4.363514970379783e-06, 4.576090579430803e-06, 3.2744849879902895e-06, 2.616506080608417e-06, 8.098085099821585e-06, 3.39789451726803e-06, 9.209765615934922e-07, 3.3300268233457617e-06, 3.7348949150220164e-06, 3.906039533959835e-06, 4.845417079285714e-06, 1.4329736731219164e-06, 5.207809570993582e-06, 1.4402567776419544e-06, 1.8388286782063624e-06, 2.9225639249944884e-06, 1.1357703149663783e-06, 3.6773513099972598e-06, 5.598197288339177e-06, 1.985729894744591e-06, 1.7140596390580548e-06, 0, 4.370349360061407e-06, 2.5238150656261714e-06, 1.9135652025336506e-06, 1.069843600025933e-06, 3.244040878977893e-06, 3.0696504130016704e-06, 2.4454667669136356e-06, 2.1802246960505697e-06, 1.7780615595658046e-06, 1.9762276824977514e-06, 2.466955429142127e-06, 1.5045294104022895e-06, 3.498501400509749e-07, 2.8117464804875066e-06, 7.974624549499013e-06, 2.6300170560943432e-06, 8.876411060052061e-07, 2.530482989776001e-06, 1.9214336730107493e-06, 1.0084453336209355e-06, 1.3805477877652252e-06, 2.8570457006332974e-06, 2.3418654376409103e-06, 1.2481158691778254e-05, 9.70939524367795e-07, 5.6426993588151995e-06, 0, 1.9520532492720353e-06, 3.226186206338465e-06, 4.24241491031818e-06, 1.3361943114878808e-06, 3.26132150378354e-06, 1.5441730451190601e-06, 2.287607840552628e-06, 1.8242437532785113e-06, 1.4721540320927929e-06, 3.0365114732121352e-06, 3.5064812609551936e-06, 3.145463554916593e-06, 1.4767485611591908e-06, 3.043444664688385e-06, 1.4488412715420339e-06, 1.6386153886508012e-06, 3.3991429401429352e-06, 6.733276675858389e-06, 8.488498222421178e-07, 1.4488412715420339e-06, 5.704942378660689e-06, 1.9313292582760625e-06, 3.0365114732121352e-06, 5.1180389997466245e-06, 6.092598734073151e-06, 9.980843752729725e-07, 3.03650906535131e-06, 3.7138225801791404e-06, 6.386247181058985e-06, 1.3467857397383833e-05, 4.824376594871601e-06, 6.137963848193983e-06, 1.2900170336129125e-05, 2.766552040664553e-06, 2.4471628331320272e-06, 2.5497006038632613e-06, 2.8199767397189502e-06, 1.8119337412787563e-06, 1.4779932008078064e-06, 1.8141606700091666e-06, 3.095143959874467e-06, 4.068443971873308e-06, 1.4282539541084969e-06, 8.93179243838579e-07, 1.637139991147405e-06, 4.6904599089851157e-05, 1.6967841134906085e-06, 7.912946421863036e-07, 1.4016000837624044e-05, 3.9671841223343195e-05, 1.6143459437792384e-06, 3.4081087655051492e-06, 1.487696357065837e-06, 8.723826132483868e-06, 7.1991261565314164e-06, 8.546653174642934e-06, 3.4766244128241664e-06, 3.8587903704104025e-06, 2.5292505387308857e-06, 2.6572468051168207e-06, 8.163011457383812e-06, 5.790216322899095e-06, 3.19843129568881e-06, 3.4152590871868043e-06, 3.1589826679744645e-06, 5.672694624559038e-06, 1.849330264444536e-06, 5.6273074783537325e-06, 2.5516605931570168e-06, 3.4885116905539888e-06, 0, 2.1650149581883016e-06, 4.172987838929642e-06, 1.422512414494812e-06, 3.1203833014575804e-06, 1.5583525976427597e-05, 2.5088271826031672e-06, 1.5097781056681354e-06, 1.8038335729640173e-06, 2.351495090004686e-06, 1.737210987468718e-06, 2.9509516481274375e-06, 3.6986605662498677e-06, 2.023912020715466e-06, 1.1835480680427886e-06, 1.748615691464776e-06, 3.0047874301495805e-06, 1.9344054562518034e-06, 2.7776436263936028e-05, 2.613510852661708e-06, 1.968411053068291e-05, 2.3987912275043224e-06, 2.001188302779876e-06, 4.996277732699348e-06, 1.153386832526844e-06, 9.0423198947165e-06, 1.0793368817844262e-06, 1.2451858272994097e-06, 2.6433422246874825e-06, 1.3047742570240898e-05, 6.444295984147714e-06, 3.0637207780918452e-06, 1.8075302809619907e-05, 4.510588799045325e-06, 4.876306476828733e-06, 9.071239290699235e-06, 4.162062118836195e-06, 2.337771201716614e-06, 1.5037851336472762e-06, 1.0817061058875737e-05, 3.381468803858182e-06, 8.361860059748117e-07, 3.3256900094331443e-07, 7.482771757666143e-07, 1.1340960292297039e-06, 1.2165828739188225e-06, 2.4903613564375026e-06, 6.723724700295442e-05, 3.146678924753016e-06, 3.1618636454922938e-06, 2.2788327953504736e-06, 3.774352232740101e-06, 5.33682157128823e-06, 1.7918450101843903e-06, 4.418821372925493e-06, 9.624216926339e-07, 6.892610766607484e-07, 4.018808266022237e-06, 8.958572472875368e-06, 1.410033251468962e-06, 1.361110644045684e-06, 4.8161480313691876e-06, 2.71849220765947e-06, 1.8373421493892514e-05, 3.877886545963095e-06, 2.8079963377319146e-06, 1.8471164981812955e-06, 2.027638756297763e-06, 4.999914108713585e-06, 4.549097809534145e-06, 1.5611507716697115e-06, 2.8412786042540804e-06, 2.3948044887677436e-06, 2.700262805795739e-06, 1.4687171714783538e-06, 2.372715921169356e-05, 8.397593560610669e-06, 1.7198806451114937e-06, 4.718614328676031e-06, 1.2868434224229761e-06, 3.9128808207857473e-07, 3.1463689567670545e-07, 3.160632337368541e-07, 3.160632337368541e-07, 1.3427618726473382e-06, 4.3920595585201913e-07, 2.6498951342897612e-06, 2.025837953546036e-06, 1.0816810166917076e-06, 8.928519768167081e-07, 1.2147909410526275e-06, 3.155083910229336e-06, 2.1541526251165916e-06, 7.184428898940041e-07, 2.7259340477338536e-06, 7.106487229174719e-07, 1.3888173940002979e-06, 1.820135203774533e-06, 4.891364697132701e-06, 6.734021153641763e-06, 1.6352435676909269e-06, 3.078969592239908e-06, 4.288621402318023e-06, 1.8994219322564243e-06, 6.532859613017128e-06, 6.471568603262687e-06, 2.1232692416030195e-06, 1.1098162463808523e-06, 2.472630699608243e-06, 1.279201845746149e-06, 2.4580924770052427e-06, 1.9299706311690543e-06, 2.0354254281470745e-06, 2.609390038697608e-06, 3.60582135714583e-06, 6.069290672802088e-06, 2.735833324416299e-06, 1.295723346043058e-06, 2.621640521381683e-06, 4.465999622703955e-06, 2.0662362214604617e-06, 0, 2.7394753041796938e-06, 3.4641375944582853e-06, 5.338074453400244e-06, 6.124557619736365e-06, 3.760445000198507e-06, 3.895466399763663e-06, 3.8033968694211827e-06, 2.204348095129052e-06, 3.608310803217568e-06, 3.8579157488874965e-06, 2.3598226904982373e-06, 2.358491155288707e-06, 2.2449186114414767e-06, 2.628853069974606e-06, 3.2414410498851585e-06, 2.5404423610039553e-06, 1.8306696091315271e-06, 2.734321021883373e-06, 1.8093634850466442e-06, 1.9373999874506744e-05, 8.730722530416756e-06, 1.959402130835944e-06, 1.8756976823812912e-06, 1.4417988836185322e-06, 1.804816515502959e-06, 9.005211007083423e-06, 3.4164928503995026e-06, 1.8768751919949722e-06, 1.256968052399827e-06, 8.946537602424234e-05, 1.60090743048193e-06, 4.811092183188537e-06, 5.432114976342164e-06, 9.691394209493066e-06, 2.101696716788023e-06, 1.2051721064194073e-06, 4.48949843582165e-06, 9.83776712861779e-06, 7.903781782380708e-06, 1.5259287813765765e-05, 9.230681998116673e-07, 1.4401666879122657e-05, 8.505100699804753e-06, 1.224030942466727e-06, 4.640200354884468e-06, 2.233414545988109e-06, 2.745159773323771e-06, 5.805932304714234e-06, 4.772950218405476e-06, 4.5224908061342355e-06, 6.446347242307627e-06, 3.300699708497465e-06, 7.2014856972551785e-06, 3.027090125691736e-06, 4.182615673330546e-06, 2.136692998864472e-06, 6.729022610288983e-06, 3.9403620349750075e-06, 3.1447828481826677e-06, 2.6089367314581216e-06, 4.942628298305343e-06, 4.0542409898282665e-06, 1.7779137696064403e-05, 5.847655962009858e-07, 2.5114707208549935e-06, 4.145923295493036e-06, 1.8573827495487935e-06, 1.0376076404296289e-06, 1.627380028834484e-05, 5.93241665405916e-06, 7.564541966375177e-06, 2.5514366293118865e-06, 2.3498409974476987e-06, 2.6527757849799926e-06, 3.9658778388111915e-06, 3.9658778388111915e-06, 3.6889278451468065e-06, 1.8141609466581388e-06, 1.2602209673213998e-06, 1.026198782994457e-05, 5.77771773259213e-06, 1.4403752074293133e-06, 3.933704721701756e-05, 3.3037817650707567e-06, 3.2746327460061354e-06, 2.078539943227943e-06, 1.413234513218437e-06, 2.7543665369341617e-06, 6.593507242213502e-05, 2.1158676596896202e-05, 3.246603730595468e-06, 2.0109322757753995e-05, 0, 2.107756032584154e-06, 2.4817329425756467e-06, 2.5037994051460182e-05, 3.3414610578506165e-06, 2.8570457006332974e-06, 2.1870876480392607e-06, 2.2321392576049698e-06, 3.0254542398426553e-06, 3.2748166164228834e-06, 6.355091848510321e-06, 5.6717184963052415e-06, 1.7191357971803703e-06, 0, 0, 2.1876792558280277e-06, 1.652436377905532e-06, 2.6064359462888246e-06, 4.076232412450403e-06, 3.794865143170238e-06, 2.4691146873040215e-06, 3.195447519247121e-06, 2.228326560940166e-06, 3.012294295605141e-06, 2.071072628522595e-06, 1.5798427284146231e-06, 4.452176569927518e-06, 0, 5.305674683883177e-06, 3.680070389690635e-06, 3.952922314462015e-06, 1.7625972562250637e-06, 1.342150629320234e-06, 8.14375394131053e-07, 1.8725044140412702e-06, 3.6445541690947985e-06, 1.8684788077400909e-06, 3.28693956451864e-06, 2.808241194947145e-06, 8.399126815954421e-07, 4.2160273067170935e-06, 1.5849483270575634e-06, 1.0870335536671733e-05, 1.485625410348573e-06, 2.3220705103747976e-07, 4.876306476828733e-06, 4.368415541884183e-06, 5.376560035197824e-06, 5.077279656711196e-06, 1.7701720123742627e-06, 2.607108029240452e-06, 2.742580306716446e-06, 1.9971840339371934e-06, 1.7613573534348461e-06, 3.820436170017988e-06, 1.0726519366107875e-06, 1.58586790635874e-06, 1.2242532345204383e-06, 5.125237738640964e-06, 3.3769702396998103e-06, 5.059471941406165e-06, 3.2236583491735098e-06, 3.4585931667221106e-06, 1.2662362540654844e-06, 1.3275542958668236e-06, 3.0344112941245187e-06, 4.205364265108118e-06, 4.2561948314727295e-06, 4.772401477459592e-06, 4.3734664715242286e-05, 2.3983827776057517e-06, 2.0780793486635937e-06, 2.7323290574742384e-06, 1.1448622588119892e-06, 2.8079963377319146e-06, 2.7537891892490504e-06, 3.1543203981097764e-06, 2.2885679639775687e-06, 2.884309326370049e-05, 6.554456789385732e-06, 2.2662372150633297e-06, 3.867337307330839e-06, 3.5095476580076685e-06, 3.4659186609750883e-06, 8.34698152039215e-06, 5.201015752632106e-06, 2.7179899801233425e-06, 4.8102714303899454e-06, 2.4688465779588814e-06, 2.97557828020558e-05, 7.195542101847626e-06, 1.384770046735096e-06, 3.1018493378762118e-06, 1.6983649304038826e-06, 1.2087579766545117e-05, 1.3060909482147778e-06, 2.9539230727231104e-06, 4.775299219347928e-06, 2.5666853653136987e-06, 6.390731165505449e-06, 4.542319492802198e-06, 2.7925285523607286e-06, 5.99132879077721e-06, 7.937829643369328e-06, 5.908259467998519e-06, 8.147641192766549e-06, 2.8187799620822964e-06, 1.5858880755788659e-06, 8.138602038222359e-07, 7.939296508854201e-07, 8.535676022367931e-07, 4.065975969167814e-06, 2.549876859575035e-06, 1.7679990808488e-06, 2.178692839657381e-06, 4.333353424383096e-06, 3.340017040899468e-06, 4.107990969273505e-06, 3.406860582477625e-06, 3.0262026650964785e-06, 1.36353897276888e-06, 9.509752831122862e-06, 3.976012130379557e-06, 3.116111611777751e-06, 1.2121555679745104e-05, 1.47487760651879e-06, 3.0868375036698887e-06, 1.4395440764405555e-06, 2.0883969622121445e-06, 2.008906911897049e-06, 4.876082507127662e-06, 2.3598226904982373e-06, 1.953631640083911e-05, 7.2364899480183955e-06, 3.8322109095390074e-05, 3.976390046592204e-06, 4.509809889133468e-06, 6.110724514903228e-06, 2.4047297116559037e-06, 1.1287399191973257e-05, 2.392710251884607e-06, 3.6598041057360103e-06, 7.170985643345631e-06, 8.427960787467045e-07, 1.4280922464496035e-05, 7.272629492042924e-06, 1.6497171084585265e-06, 1.366513265786966e-06, 3.241485438410915e-06, 4.469912542691877e-06, 5.041299473565484e-06, 1.7968910029916368e-05, 9.250262252745418e-07, 2.238870659582119e-06, 8.556691331529823e-06, 5.104367748923067e-06, 3.349125262088868e-06, 7.521401607595145e-06, 2.2788327953504736e-06, 2.0874922621369394e-06, 3.6087017744755596e-06, 1.4014568453070665e-06, 1.6682563241112412e-06, 1.345792345201674e-06, 0, 3.494584655465652e-06, 2.3025555409205167e-06, 2.2039688494108707e-06, 1.5925920446205344e-06, 1.3711856702455688e-06, 3.0383442700153004e-06, 5.346744588423296e-06, 2.9659162422772395e-06, 8.875735638816081e-07, 6.870982579839828e-07, 8.503977331309562e-07, 4.6038322229656625e-06, 1.6818599354720673e-06, 4.19413435233411e-06, 1.166453109826147e-06, 1.9986154255014246e-06, 2.3970507893609055e-06, 3.7533002937901274e-06, 1.8540538416996203e-06, 3.3138214988008855e-06, 5.13111224869885e-06, 7.149722758232788e-06, 8.452906384681778e-07, 1.1247896303798311e-05, 1.2939686295239454e-06, 1.8260706562417388e-06, 1.4320636509518614e-05, 3.375142288079572e-06, 1.8804217928238019e-06, 9.088019117005496e-06, 2.77373086217935e-06, 9.5958955157635e-06, 9.800318053938668e-06, 2.6103193796888643e-06, 2.730879658629256e-06, 1.6823164182907326e-06, 4.579515051473856e-06, 1.4893668621680856e-06, 1.691537629857591e-06, 3.2241832124764794e-06, 1.0121759906461585e-05, 1.9508792080801926e-06, 1.2323227184710118e-06, 4.608535039626482e-07, 4.465999350288382e-06, 5.753870570061783e-06, 1.9881710312790206e-06, 9.558737803123618e-07, 4.9166894207254795e-06, 1.56393000163906e-06, 1.4262841545331229e-06, 1.4739124926426187e-05, 1.659379506298262e-06, 2.2788327953504736e-06, 7.5281115059159955e-06, 1.8420067358394692e-06, 3.0867240610850315e-06, 5.094888498543406e-07, 3.0587747016352945e-06, 2.7179838458607924e-06, 5.764785896076791e-06, 4.891364697132701e-06, 7.893846434511708e-06, 2.5290777528252488e-06, 9.09710637248234e-06, 5.65099330800282e-06, 3.257663338490307e-06, 5.023058304134635e-06, 3.068821793825296e-06, 1.044271816404397e-06, 1.8308877628085001e-06, 1.6540464007337818e-06, 8.103879397813109e-07, 1.4473357304757048e-06, 8.367933309608784e-06, 1.826070600548708e-06, 4.86402506566441e-06, 2.8898654568263002e-06, 2.6419376530506324e-06, 1.62569979559222e-06, 3.18929865672359e-06, 3.053579139701462e-06, 3.159266947565255e-06, 3.0521888299995978e-06, 1.9184047249400604e-06, 4.865036601410818e-06, 5.416371545376174e-06, 4.559654098461927e-06, 2.015025620590544e-06, 1.1656178836658684e-06, 4.813452223926828e-06, 2.2110311579850967e-06, 1.0919391682734507e-06, 3.3565849933922087e-06, 2.3345238508965744e-06, 2.843831847586953e-06, 2.296090426582351e-06, 1.9029118929226204e-06, 2.355178029960303e-06, 2.749117418805963e-06, 2.2788327953504736e-06, 2.2632246895845886e-06, 9.522170138429436e-07, 2.81142928349814e-06, 5.659124633681089e-06, 2.0076758371051365e-06, 2.13887432704796e-06, 9.059737139860092e-07, 1.961535539775966e-06, 2.8197642268737553e-06, 7.481584444877514e-07, 1.4457306448478812e-06, 2.1876792558280277e-06, 1.129757460248941e-06, 7.374495465990215e-07, 1.0919391682734507e-06, 1.1732858878508929e-06, 3.3602458637699738e-06, 8.251700008306258e-06, 4.6809927491058914e-07, 1.0919391682734507e-06, 2.604156520296277e-06, 1.665075907404492e-06, 4.1950065118330485e-06, 1.8611651029879389e-06, 2.032242443456743e-06, 9.474443385499931e-07, 1.521257282603625e-06, 5.180544909840684e-07, 2.0710067481557296e-06, 3.4539024242678254e-06, 3.6630071522069382e-06, 1.3554602321477445e-06, 2.212813411559792e-06, 1.615023421059359e-06, 1.37550285799191e-06, 2.271886868161963e-06, 1.6289889144180082e-06, 1.069843600025933e-06, 4.876306476828733e-06, 1.8494345272611619e-06, 1.6556429951218299e-06, 5.368026960349509e-07, 1.1157822477098895e-06, 0, 6.339204596963355e-06, 2.731555046535668e-06, 1.9414751362799715e-06, 2.0249387387162777e-06, 5.482945987764616e-06, 7.486388254821894e-06, 7.438746579843129e-06, 4.299719426001789e-06, 2.51337249989927e-06, 2.9907685038910236e-06, 2.47050197139718e-06, 5.0832027964928456e-06, 1.8027747659076015e-06, 8.645489588188763e-06, 2.79409917432623e-06, 2.7772670094819937e-06, 1.44580670745189e-06, 1.896550652852161e-06, 1.5576106017698995e-06, 2.6585624575821106e-06, 7.152362822231116e-06, 1.0199465182961368e-06, 1.2122715832995934e-05, 4.117222530290564e-07, 1.5667370675223022e-06, 1.138759384490466e-06, 3.0743180511805687e-06, 5.4990454674222155e-06, 2.2427389833377355e-06, 3.464137277501801e-06, 2.0120269605833083e-06, 4.235837944764322e-06, 2.4029620990330715e-06, 1.8241778520717657e-06, 1.5255665178559478e-06, 1.2948619018195082e-06, 3.054458093202384e-06, 3.2996382816631963e-06, 0, 3.504062710348186e-06, 2.180599689753099e-06, 1.970007530505915e-06, 2.708296674724561e-06, 1.576542151211002e-06, 1.2112207707951957e-05, 2.3963367839931935e-06, 1.5925920446205344e-06, 4.8216682650623945e-06, 2.9883640412171894e-06, 1.2684237299525739e-06, 9.439249808871308e-06, 1.2330259573585784e-05, 2.3951198543415135e-06, 3.0806069323030556e-06, 7.055609952702877e-06, 2.4735278546523498e-06, 3.185312694638506e-06, 4.629917430798689e-06, 4.68125437297671e-06, 4.698873273483313e-06, 5.598453572703773e-06, 2.8158302162432655e-06, 3.9555358739676975e-06, 5.63512817036419e-06, 1.5638439567308102e-06, 2.8322919194564567e-06, 1.183466257383433e-06, 5.212368141261406e-06, 1.6187637401653718e-06, 6.533434558668168e-06, 6.807553196168446e-06, 4.6475214136589935e-05, 3.880563943213525e-06, 7.932739442213663e-06, 1.6653623982670665e-06, 1.5387082413528897e-06, 1.4729567753486433e-06, 1.502176499914845e-06, 3.3884414415135373e-06, 1.9932514254695475e-06, 2.561066258299788e-06, 2.243649002893495e-06, 2.2303937912025564e-06, 1.5286111108096751e-06, 3.921217125871458e-06, 3.481208057407263e-06, 5.9846995209081595e-06, 1.4720329445080103e-06, 5.020834905773709e-06, 3.697436497855745e-06, 3.765530610674749e-06, 2.008587601557365e-06, 5.227808625300057e-06, 1.0127710932610515e-06, 1.0469394248862607e-06, 4.759355985767789e-06, 1.0150657581924202e-06, 2.6471668874512997e-06, 1.9111515223834765e-06, 1.0495114464173773e-06, 2.3361586826767548e-06, 2.5305070626170383e-06, 1.6994431403221935e-06, 2.0134971154179355e-06, 5.1245991323865145e-06, 4.067590371702899e-06, 2.2788327953504736e-06, 3.34471501851785e-06, 2.9962463664339007e-06, 1.2980010521315745e-05, 2.5917776635900007e-06, 2.452337260516913e-06, 2.3075204753068997e-06, 7.951437203696959e-07, 1.7498932959855494e-06, 7.34764032610517e-07, 1.006486670856789e-06, 7.454145401292192e-07, 1.1608286077787423e-06, 1.514935096367575e-06, 2.086495552916023e-06, 2.9085507855511407e-06, 5.7593456277026e-06, 1.76737978828991e-06, 3.1634291431690966e-06, 2.9192552436785336e-06, 2.060835105267191e-06, 1.3631829153254153e-06, 0, 5.709514788753754e-06, 5.463591613642707e-06, 6.3278975349896874e-06, 2.5485154616899e-06, 2.929259947562475e-06, 2.6168235857324874e-06, 2.913173583588732e-06, 1.8742114720132515e-06, 1.3563323930611368e-05, 2.536185199738067e-06, 2.0476443833678404e-06, 3.275090193960562e-06, 2.760585378833102e-06, 1.4671169659829207e-06, 5.065084456894171e-07, 1.382367905888267e-06, 6.256628083449575e-06, 3.915672363563446e-06, 4.267523242898198e-06, 2.5982572229499443e-06, 3.449357730763626e-06, 3.135520978962469e-06, 2.570324913927822e-06, 2.27751882671737e-06, 2.7720994513810843e-06, 3.935123843262528e-06, 3.713539252912431e-06, 3.910645536838223e-06, 4.333353424383096e-06, 5.190934293761682e-06, 3.1268893394840205e-06, 2.3043065711469606e-06, 3.4176168134040275e-06, 2.7899569983848706e-06, 9.727007405314142e-07, 1.7254642762866017e-06, 2.4401749932390947e-06, 2.9297314275194813e-06, 3.3672527884542915e-06, 2.0281406550679476e-06, 3.141408829078209e-06, 3.833495384824451e-06, 2.589403650220103e-06, 0, 4.582123640368437e-06, 1.33574484767805e-06, 3.834447500943572e-06, 2.5615990989105186e-06, 2.452381918836657e-05, 2.222844788562407e-06, 6.687054732429167e-06, 1.3464767820716996e-06, 5.471662532117636e-06, 2.851699209643838e-06, 1.1177945220328597e-06, 9.426917608346292e-07, 0, 2.4580829559655324e-06, 0, 1.314909576415903e-06, 4.127079424798497e-06, 3.6425455929716832e-06, 1.5698856820030073e-06, 2.494012432248841e-06, 1.0073803712773073e-06, 7.75635863396796e-06, 6.471569195388666e-06, 5.076521136429331e-06, 2.191483092186265e-06, 1.520744362131522e-06, 2.9889190085430695e-06, 2.2908891431138112e-06, 3.1493528740408285e-06, 3.983170884902332e-06, 5.920660874231136e-06, 2.68285371663839e-06, 3.556707030351299e-06, 2.5176322560834e-06, 3.779561537969978e-06, 8.395603512504786e-06, 2.3310120499790644e-06, 8.647219484833612e-06, 5.592173632844209e-06, 2.54020071715739e-06, 5.03886913557373e-06, 3.144395282794827e-06, 5.237610429857584e-06, 3.000466799468973e-06, 2.178692872881178e-06, 1.0919391682734507e-06, 2.1402663340042154e-06, 1.978963360645807e-06, 7.814651698895966e-06, 3.001513041193978e-06, 6.800566965839739e-07, 1.8289308932366124e-06, 2.0996719303555662e-06, 4.55505347885072e-06, 4.289104935152575e-06, 3.648731641743231e-06, 1.5011210725677248e-06, 1.2682054080717025e-06, 3.0879138622535667e-06, 1.1593426657089462e-06, 1.8402250364033964e-06, 3.1201054706708125e-06, 3.6071515744171686e-06, 8.680502955603379e-06, 4.897470601924068e-06, 4.523606320373696e-06, 5.0125496397949904e-06, 9.914607612139724e-07, 2.7253670236575946e-06, 3.268982346753054e-06, 3.2079500636688154e-06, 7.990569972781657e-06, 4.70887334676313e-06, 8.570708774801555e-06, 6.173997147992687e-06, 2.2788327953504736e-06, 2.1960806596963207e-06, 1.525979184003581e-06, 1.5838012692344581e-06, 1.4105498286074552e-06, 3.989463702229168e-06, 1.2974807586598697e-06, 6.757531964343225e-06, 3.364230018815877e-06, 4.1501246891667295e-06, 2.5920447217755754e-05, 1.9369541278010297e-06, 3.2232467514682743e-06, 1.575648499072226e-06, 3.694511676629655e-06, 2.5808668651451443e-06, 3.065305461637199e-06, 2.2033445453258483e-06, 2.649110541105909e-06, 6.800567069544439e-06, 3.1154635690305664e-06, 1.7057939913236328e-06, 1.4767485611591908e-06, 1.2741235686529433e-06, 6.601904837108138e-06, 3.5719100535298996e-06, 1.7689512207194819e-06, 6.020200172818711e-06, 2.406542441988216e-06, 2.334485438671018e-06, 2.8684401128406994e-06, 4.171977681825619e-06, 0, 1.0817410931736814e-06, 1.5379344390932537e-06, 7.614202344860737e-06, 3.346411669616368e-06, 3.99018529431907e-06, 3.559503432392006e-06, 1.4884221618998656e-06, 2.0527599541441466e-06, 5.197401286027334e-06, 1.6174541769242108e-06, 1.940725233445592e-06, 1.7288312196382962e-06, 1.7701720123742627e-06, 2.2343730151647193e-06, 3.5605152534776086e-06, 5.0289617101156275e-06, 1.588623233116011e-06, 1.188650983774635e-06, 2.2214346763951506e-06, 4.703344742288797e-06, 8.297543878163299e-05, 1.4843261299368321e-06, 1.4834139663944547e-06, 2.1929959170067453e-06, 2.2740452571537957e-06, 2.8338011489965887e-06, 1.6129714261368805e-06, 6.2977031128108944e-06, 5.557777892120151e-06, 1.4531040029580872e-06, 2.158225251307563e-06, 1.8865979139701454e-06, 2.3665252129520346e-06, 4.357714385545801e-06, 2.747343910225265e-06, 1.5955094830220527e-06, 1.8170026121917912e-06, 1.0538124017396956e-06, 1.7006381102128285e-06, 3.636833875779585e-06, 2.924648773612553e-06, 4.86402506566441e-06, 3.853024762143254e-06, 2.564440702695108e-06, 1.9748411147477535e-06, 9.804846512449542e-07, 1.0767712051999136e-06, 3.346411669616368e-06, 3.184365687771406e-06, 2.6776236041941294e-06, 2.8380862701351707e-06, 1.3376683629750378e-06, 3.108185768854206e-06, 1.6585199808267932e-06, 2.980272297677274e-06, 8.672600070889066e-07, 1.386259337735814e-06, 4.2405307174436376e-06, 2.81248440615583e-06, 2.2509581175120506e-06, 2.016097082355806e-06, 2.112000214474585e-06, 1.3066567968417846e-06, 2.424109636643061e-06, 9.313524747604629e-06, 2.5277680493927685e-06, 1.5847738809161919e-06, 3.884050462570634e-06, 5.938199880543492e-06, 5.33941214597402e-06, 5.596851006382078e-06, 7.819499204835832e-07, 3.0293700627318748e-06, 9.809927560132889e-07, 7.177598773430177e-07, 1.74937160479237e-06, 1.8300540668228145e-06, 2.1188567414006473e-06, 2.885346172526202e-06, 2.4581594251998093e-06, 6.348266553676304e-06, 2.4378572756147025e-06, 3.866432742079916e-06, 1.8133580676695695e-06, 3.575271688537975e-06, 8.569762549382183e-07, 6.3762298788841e-06, 3.959977407342548e-06, 1.6585460565324832e-05, 3.19984723189709e-06, 4.976372992613766e-06, 1.2514852794266897e-06, 3.6143999548629793e-06, 5.577444679365804e-06, 4.260646804958994e-06, 1.3684683379709396e-05, 3.0620687386671026e-06, 0, 1.6177702681359477e-06, 1.1724766646735223e-06, 5.662976576838382e-06, 2.4563966645683757e-06, 2.8011914567979837e-06, 3.859369098548707e-06, 4.996277732699348e-06, 6.759881585036097e-06, 5.874888708174539e-06, 1.9214336730107493e-06, 4.890045742577896e-06, 1.9313539683669158e-06, 3.805630567316576e-06, 3.667943695163376e-06, 4.445566712879318e-06, 7.083598753956664e-06, 5.781746588185162e-06, 2.470613940225757e-06, 7.80516553956261e-07, 3.094921290182172e-06, 1.514532946035414e-06, 1.1626463214842006e-06, 2.2519851051200545e-06, 5.096016327129693e-06, 7.4250109926352215e-06, 1.1764695591829124e-06, 3.83209415508958e-06, 1.6593089080426963e-05, 3.0221003294297075e-06, 2.3070704690510827e-06, 3.0214702174288884e-06, 2.8570457006332974e-06, 1.5196435845727508e-05, 1.4450526078730124e-06, 2.3276294119804157e-06, 4.232538990081178e-06, 3.006249170696297e-06, 3.195447519247121e-06, 2.199520523111837e-06, 4.827430672072522e-06, 3.4463848112867806e-06, 9.95284291778356e-07, 4.739895852731837e-06, 4.162101596758266e-06, 3.8038697694801966e-06, 1.0683542760838473e-06, 4.925347541618659e-06, 1.5011210725677248e-06, 2.5120923053028486e-06, 0, 2.132507702139565e-06, 8.428595450535145e-07, 7.857248462825892e-06, 3.512574688048156e-06, 1.7818396194879365e-05, 3.0111746823451104e-06, 1.5621309606158797e-06, 7.584898410715456e-07, 2.6917532130081233e-06, 1.2105433732092567e-06, 1.3972990638826461e-05, 5.319474626408678e-06, 9.96267611832041e-07, 1.160733233529733e-06, 1.088243688040719e-06, 2.947478275960622e-06, 4.8235013510541356e-06, 1.998615577890049e-06, 0, 0, 1.7653806323375034e-06, 3.0470742459691663e-06, 4.333408866797358e-06, 1.366513224109901e-06, 7.650709673644592e-06, 0, 9.881272118856398e-06, 3.869472542821241e-06, 6.4315143640046495e-06, 6.962821210757633e-06, 2.9215810225960872e-06, 1.3485151547574122e-06, 1.337197848550031e-05, 3.4367064814408796e-06, 5.679064988468104e-06, 3.5111489774513494e-06, 1.3618565959431018e-06, 1.1722588042651942e-06, 3.181760359462056e-06, 1.179955758789443e-06, 1.8559491736292454e-06, 1.1226667869157077e-06, 4.507137170774531e-06, 5.7083907656968545e-06, 2.2312111097322357e-06, 1.0196673073418335e-06, 2.628783918255021e-06, 3.905073035466969e-06, 8.922360075231477e-07, 1.419289876703963e-06, 1.608605251487653e-06, 1.6478666003836978e-06, 1.0398165528961017e-06, 4.706591914550826e-06, 0, 1.655344040120409e-05, 7.28553364690326e-07, 0, 2.9626982629557635e-06, 7.915005765666749e-07, 2.7394753041796938e-06, 3.385477967505639e-06, 4.88149605354461e-06, 2.1737572623373466e-06, 4.011146132118945e-06, 2.3964576704344074e-06, 2.844805887725733e-06, 2.4503559605124064e-06, 2.3387500608215615e-06, 2.3188066380206895e-06, 3.0914362351819406e-06, 3.1421432933300343e-06, 1.7322345295000693e-06, 4.114274186571954e-06, 1.3201977243235878e-06, 2.883815559821746e-06, 1.3053977771788037e-06, 1.6032816600499651e-06, 3.0636791976093265e-06, 0, 9.400691840707158e-07, 4.313010474983479e-06, 2.0354254281470745e-06, 1.1096006210546817e-06, 1.3152348135852766e-06, 1.850925448509907e-06, 1.060458234690066e-06, 3.2767114024381794e-06, 2.1601141546834465e-06, 2.7769165712183775e-06, 3.237815552829501e-06, 3.0784817005623985e-06, 1.8802799699461564e-06, 2.437801103468122e-06, 1.670790130857639e-06, 0, 3.713539252912431e-06, 1.6020550041097421e-06, 4.621533234763838e-06, 3.7941281310956405e-06, 2.7480261357719395e-06, 1.3601848820589153e-06, 5.968939701214383e-06, 4.678209163116622e-06, 4.408014171429797e-06, 3.5827755097988747e-06, 2.0014012004022417e-06, 1.9230893506878504e-06, 1.739161773747427e-06, 2.038613709040332e-06, 5.094888498543406e-07, 3.88123198327512e-06, 4.273272065940429e-06, 3.5509841368876325e-06, 5.0267367846237535e-06, 3.7356242946951495e-06, 0, 8.40786143138731e-06, 2.8133675813036178e-06, 9.515342314438718e-07, 5.972637102180693e-06, 4.881155055924208e-06, 2.0459836480783607e-06, 3.7881463742489744e-06, 3.983224823310875e-06, 1.9971840339371934e-06, 1.393622441332765e-06, 0, 1.574848675065024e-06, 3.234983863027134e-06, 4.6356680573981165e-06, 7.124215738968257e-06, 0, 3.982870471048798e-06, 2.5824962751693905e-06, 5.278350331772572e-06, 3.0038361022930664e-06, 8.774724771274394e-07, 3.8155277063898255e-05, 1.1789265968414237e-05, 2.0487879576453714e-06, 4.730299481129863e-06, 4.546292128656064e-06, 6.427307181129672e-06, 5.201455558849584e-07, 2.92306686681103e-06, 1.7498932959855494e-06, 2.0897901526865437e-06, 1.876484724210277e-06, 3.3467721708304577e-06, 5.2252221580992025e-06, 3.2287775995880967e-06, 1.6570919822307912e-06, 0, 4.646547674173197e-06, 2.2308807886456723e-06, 3.5668921878152017e-06, 1.03990906440265e-06, 8.535676022367931e-07, 2.502293369596679e-06, 1.424387837179792e-06, 2.156544283977646e-06, 2.4716489502114348e-06, 2.007469098651393e-06, 3.2804253242980526e-06, 1.0768671851648945e-06, 5.192361880239589e-06, 1.3904831617397449e-06, 4.4947556461706034e-06, 4.345918295694198e-06, 1.4708660174304226e-06, 1.3663409003162227e-06, 2.1679008913889054e-06, 4.5367842088920475e-06, 1.5525508995854379e-06, 1.4767485611591908e-06, 3.164648510892494e-06, 5.027135469339596e-06, 1.5388046830834689e-06, 2.5731522319568053e-06, 2.105655014216775e-06, 7.426431675729732e-06, 2.2137931188819054e-06, 1.7288576889663967e-06, 1.860623545706509e-06, 2.3140474149228327e-06, 2.3391726529218606e-06, 3.1276635250939676e-06, 1.8305318735768915e-06, 3.7273184381013293e-06, 1.4094226570898625e-06, 2.7413665444497723e-06, 7.401053845776938e-06, 3.017264319871342e-06, 3.123916892308545e-06, 4.48117027594275e-06, 5.200560677613103e-06, 2.6730503466506614e-06, 5.742395217643022e-06, 4.001622704004481e-06, 1.1474598461382562e-06, 2.650737480022268e-06, 1.0782842332298494e-06, 3.3381178701666953e-06, 1.746833886416883e-06, 2.589692157582372e-06, 1.92975724419926e-05, 3.599728378127631e-06, 5.046893343769023e-06, 1.0701084496504751e-06, 2.9151175686858752e-06, 2.9816323037901733e-06, 2.7052216993642026e-06, 6.934526240773278e-06, 1.2207012645333951e-05, 3.299267260799275e-06, 2.18929027638125e-06, 4.4179460906133905e-06, 2.3621549744306467e-06, 3.4365787660653538e-06, 6.08499686862045e-06, 4.427976486874424e-06, 1.5667583313946124e-06, 1.498773629460135e-06, 1.9986154255014246e-06, 1.5791016493826169e-06, 9.636681708035009e-07, 9.674886753938171e-07, 2.089790280158887e-06, 2.1936184596706644e-06, 2.3490100890135796e-06, 2.2147203705031678e-06, 1.8926223562026186e-06, 1.8401464634680556e-06, 1.8409846729051207e-06, 3.2559759338874035e-06, 1.5667583313946124e-06, 1.3178950755896674e-06, 1.5170938865394856e-06, 1.7128771793792533e-06, 6.386247181058985e-06, 2.0279060181663807e-06, 5.866123346562091e-06, 4.791155432856118e-06, 2.0599249423885683e-06, 1.68115590823566e-06, 3.1600227903127323e-06, 3.6172551762790322e-06, 3.4866491043669732e-06, 1.1028658321917674e-06, 3.4397580727998126e-06, 2.086829348795805e-06, 2.8751148979189473e-06, 1.0962013786812093e-05, 6.4521611946034715e-06, 2.9459254402741617e-06, 2.6030040985482177e-06, 3.6371997057112775e-06, 1.2013906039486927e-06, 3.406860374667065e-06, 1.816198281717799e-06, 2.4535524686554956e-06, 3.608310693168286e-06, 2.2657896191378505e-06, 5.704164938967759e-06, 5.65476314208613e-06, 1.010585463984861e-05, 2.469302844331152e-06, 3.1310126032049128e-06, 4.049186440431338e-06, 3.236297485458035e-06, 2.1058919994590448e-06, 1.011683080318109e-06, 5.1657522500308235e-06, 2.383680964138146e-06, 2.716175174086274e-06, 1.0610076537686869e-06, 4.507137170774531e-06, 2.3267373038820124e-06, 5.7083907656968545e-06, 2.1229327248836965e-06, 1.484967792887469e-06, 1.0856343568915942e-06, 1.5694389802038657e-06, 2.1889739725333052e-06, 1.6766052663383566e-06, 5.448322561465446e-06, 2.3820062654953745e-06, 3.4320130092567725e-06, 1.4917138120309309e-06, 2.5513011166081238e-06, 1.5554461498339804e-06, 1.5627649817232791e-06, 1.43587514531049e-06, 3.19843129568881e-06, 3.411936236419257e-06, 2.130598199390203e-06, 6.451106456136099e-07, 6.772148165867247e-07, 1.7511235798728094e-06, 6.608479783979218e-06, 1.4360027185940522e-06, 6.924017306316269e-06, 4.70509375250919e-06, 3.7496877168378145e-06, 4.638696327841197e-06, 0, 4.427419920974827e-06, 0, 5.813891218876492e-06, 1.4581127253447138e-06, 7.64091284936952e-06, 2.298257837831145e-06, 2.121441413788212e-06, 2.5253784180354003e-06, 4.765603427374041e-06, 9.20297963679174e-06, 2.8923948323836715e-06, 1.7140595867812297e-06, 2.9888780786327194e-06, 1.7938222696326047e-06, 3.228356642639328e-06, 8.877930471920931e-07, 6.026495102981129e-06, 6.8218172657040876e-06, 1.7384263371255223e-06, 2.774013045040797e-06, 4.218907273390881e-06, 1.6653624998503031e-06, 7.0493685001246895e-06, 2.2826590868028994e-06, 4.724817470574969e-06, 2.5779123170184895e-06, 2.3970507893609055e-06, 8.43890133505435e-06, 3.0262026650964785e-06, 3.836940843844889e-06, 2.248429658375662e-06, 2.189348735003511e-06, 3.4365787660653538e-06, 3.9023792042267936e-06, 5.640552944154294e-06, 1.0469161478580036e-06, 4.696920123054018e-06, 3.0622697195426773e-06, 3.1014037907026486e-06, 1.370802489272377e-05, 4.237615688018653e-06, 7.069391007934988e-06, 2.501252240269763e-06, 1.0084502546585196e-05, 5.918040252774642e-06, 1.6735237701929664e-06, 1.2249197925495145e-06, 2.823481855092796e-06, 1.4437891634061858e-06, 3.346411669616368e-06, 1.3284675932276923e-06, 1.8230530306396029e-06, 3.51838950038363e-06, 1.3247002311581814e-06, 4.168624653147062e-06, 4.328447565733405e-06, 3.677984423039912e-06, 4.9101491373920505e-06, 3.2471625352118783e-06, 6.532859613017128e-06, 1.0555327374723777e-05, 5.135097793442987e-07, 1.8396676339608337e-06, 2.2507565654246274e-06, 1.7352569911646154e-06, 2.199354633559175e-06, 3.543455371437159e-06, 3.996793681872558e-06, 3.5969509473798804e-06, 4.140627777768116e-06, 2.1557543743024793e-06, 4.633113909109453e-06, 2.441586079554047e-06, 2.6724581200846672e-06, 1.234558460758943e-06, 2.739807981931152e-06, 0, 0, 5.558862834880234e-06, 5.179263999399152e-06, 2.3889973641784577e-06, 3.092437281868056e-06, 5.859774587182074e-06, 8.10958509133625e-07, 3.9923197437040885e-06, 2.630467367293803e-06, 2.235340894789255e-06, 1.3331673987228684e-06, 4.363414095289856e-07, 1.6064334329340386e-06, 4.671801661372137e-06, 2.0229172574837888e-06, 4.689452633900823e-06, 6.761433885811742e-06, 1.439524604961907e-06, 1.9341350323727456e-06, 2.2872773668787268e-06, 4.528790416157562e-06, 2.3064992188865117e-06, 3.2000887314715284e-06, 2.2068444148211685e-06, 3.707782140480425e-06, 4.165845321844007e-06, 0, 3.878846374999278e-06, 1.5054735897336555e-06, 2.1712964163415724e-06, 1.8399336318517025e-06, 1.0852830773151409e-06, 1.5441730451190601e-06, 5.061940314186177e-06, 1.5441730451190601e-06, 2.28578779348702e-06, 2.18929027638125e-06, 2.1930237074121274e-06, 1.1884626670983921e-06, 3.4853361656347005e-06, 2.576158434361415e-06, 2.17014571651033e-06, 1.0635042418392395e-06, 2.106311798533031e-05, 2.4892526813384488e-06, 1.5249058702232846e-06, 2.896888402366404e-06, 1.980716122536394e-06, 3.2922083144550793e-06, 5.985141144555847e-06, 2.6784200730807836e-06, 2.057057550728158e-06, 2.7818642763962086e-06, 0, 7.117190513441664e-06, 6.026000146967198e-06, 1.14047175479978e-06, 1.1368929523798379e-06, 1.8407662424926157e-06, 1.4767485611591908e-06, 1.5154712934469696e-06, 7.408020016023156e-07, 9.99761862940942e-07, 1.7994857456152387e-06, 4.026553806544347e-06, 1.5780867226995926e-05, 1.1646142776070978e-06, 5.34050601223071e-06, 1.9917130129734934e-06, 1.0885948966667062e-06, 1.317668218576357e-06, 1.2791128965344383e-06, 2.133688974923652e-06, 2.3107662768998747e-06, 1.1074007701934107e-06, 6.026500892718305e-06, 4.650260299826644e-06, 1.8330833927694752e-06, 0, 1.6114596730986388e-06, 2.185201416203935e-06, 4.350899555499245e-06, 7.732496829896475e-06, 1.8392834764407065e-06, 1.9598835227429404e-06, 1.843131081452538e-06, 4.675890475514349e-06, 1.8123689531474106e-05, 1.4434980847027335e-06, 2.158223309519308e-06, 1.8642460982414207e-06, 1.137384491975458e-06, 2.1851469003726804e-06, 3.964371312169016e-06, 1.5474555755899996e-06, 2.4213160559578214e-06, 4.763590510481215e-06, 3.720412419651051e-06, 1.8472130023944673e-06, 2.57882191348287e-06, 4.063246859518993e-06, 1.7323889926226054e-05, 5.396200556158795e-06, 2.4609200570779404e-06, 7.745381276378172e-05, 1.7397428973893623e-06, 2.087962552817924e-06, 2.3403963462819266e-06, 1.686881223749935e-06, 1.7933783023368016e-06, 2.2744890153238186e-06, 2.0684885070086093e-06, 1.649815375512196e-05, 1.0371181752008368e-05, 1.0139938353167496e-06, 7.040351470206973e-07, 3.904107133333698e-06, 1.9802053955430344e-06, 4.762013786624208e-06, 5.799225689433573e-07, 2.7308792838303408e-06, 9.20171778523706e-06, 2.980613725823276e-06, 4.474162788250843e-06, 4.052099016808019e-06, 7.2677820616785184e-06, 2.757702965565904e-06, 3.23848677411051e-06, 4.171977681825619e-06, 1.4239345033812124e-06, 9.898452315965828e-07, 2.798035799790286e-06, 5.998591441598692e-07, 2.2531001822638966e-06, 2.980168255200977e-06, 2.0441468207114912e-06, 4.576349701638101e-06, 6.052862179800904e-06, 4.821898412059509e-06, 2.023591744849159e-05, 1.8826255786228998e-06, 2.2762474079162614e-06, 2.4542839709549204e-06, 4.716110704097134e-06, 5.219174082622776e-06, 9.884918155425355e-06, 0, 3.3057117223862165e-06, 1.3559336573556437e-06, 2.613510493971046e-06, 1.935740638100219e-06, 6.69355524599638e-06, 4.478358821414918e-06, 4.203710266743667e-06, 4.386817246640159e-06, 5.0622189193697436e-06, 0, 3.7532523878593606e-06, 3.4337446752256845e-06, 3.043968176686304e-06, 2.523097852212448e-06, 3.604669634346538e-06, 2.474731304022352e-06, 4.64564130477755e-06, 4.5693915431209455e-06, 2.0827173634084172e-06, 1.0221950561966e-05, 2.631934150491777e-06, 9.158278522992232e-07, 1.485369703579456e-06, 3.550221130827968e-06, 6.502258694587022e-06, 8.111778859953876e-06, 3.205693424370885e-06, 4.38854952553344e-06, 1.2221845382285093e-06, 5.218361279788456e-06, 1.7934893385576843e-06, 2.2068444148211685e-06, 4.37777295615058e-06, 2.884887642513744e-06, 2.780926064148566e-06, 1.0727180058540219e-05, 1.9341350323727456e-06, 7.091087143856231e-06, 2.6277170476079875e-06, 2.313491944546859e-06, 1.231955911649376e-06, 6.51331898943273e-07, 6.011896491135011e-06, 1.274325419777159e-06, 3.2922083144550793e-06, 1.0294289607885954e-06, 2.530004686820257e-06, 1.557495548154017e-06, 5.046692963861597e-06, 2.283698448303321e-06, 1.0146027891770454e-06, 5.29999916180222e-06, 5.246586919647499e-06, 1.5172081767542463e-06, 3.857581544613254e-06, 4.457756520363583e-06, 1.0452790830175154e-06, 1.3835780865404819e-06, 0, 2.3245515421057312e-06, 4.541604417494844e-06, 2.622413179627408e-06, 1.1315269345118831e-06, 1.875331481493774e-06, 3.1345284993341807e-06, 2.331442843217194e-06, 5.047258337319703e-06, 4.476596542228276e-06, 2.74755352089255e-06, 3.0587747016352945e-06, 5.27708700361591e-06, 0, 0, 5.029583770969965e-06, 2.864086275429273e-06, 2.3837366162529062e-06, 1.6013457998286712e-06, 2.260074307030831e-06, 8.747582265648546e-06, 2.4922024124917847e-06, 1.869658514182062e-06, 4.346603676001642e-06, 1.633442498189556e-06, 3.978702609515133e-06, 9.457260693693254e-06, 3.257663338490307e-06, 1.6849731230339452e-06, 1.4083261089797435e-06, 2.803818858438929e-06, 3.2037564543782654e-06, 3.479024714844201e-06, 2.2752701086515314e-06, 1.2368267765793651e-06, 1.91102116594376e-06, 7.860397224037262e-06, 4.366527458804819e-06, 2.5141602911536984e-06, 0, 1.8330817994231878e-06, 2.196080793652138e-06, 1.5287685575656067e-06, 2.9815944290544252e-06, 3.0768619590589475e-06, 2.953047695995257e-06, 3.1279662125272e-06, 9.422034653138622e-07, 0, 2.4548892679538524e-06, 1.4822586315444441e-06, 4.300512022132025e-06, 3.145744795020545e-06, 1.6145049335985685e-06, 2.1119980888284915e-06, 1.6810338309408338e-06, 1.3007246174042284e-06, 2.8440457405991264e-06, 8.977168111381852e-07, 2.6302153487740835e-06, 0, 3.3962134031816524e-06, 3.244567180728221e-06, 3.4159494868389556e-06, 1.5230699160403556e-06, 3.106495919998572e-06, 3.3748978193867056e-06, 1.8742932715759212e-06, 1.1505239092902636e-06, 1.0201179954998019e-06, 1.6810338309408338e-06, 2.8366681499105704e-06, 1.8043283341205003e-06, 5.190836771123076e-06, 1.2992065964261522e-06, 1.2662061318119393e-06, 2.6138642665904284e-06, 1.9275002215207178e-06, 0, 1.4846733478641876e-06, 6.0269558180274385e-06, 1.0426388229116628e-06, 6.908942051945221e-06, 3.8050777208626894e-06, 3.464137277501801e-06, 4.757849662289373e-06, 3.285832430150895e-06, 3.162488545921704e-06, 4.224318106778369e-06, 2.1737572623373466e-06, 1.8927290020446059e-06, 1.4533877100539541e-06, 5.004039702802436e-06, 2.7525556491850353e-06, 2.3069476801473986e-06, 4.646655378289563e-06, 3.2656155622525624e-06, 8.571003352105217e-07, 7.98297855402469e-07, 5.617174354524611e-07, 8.377715821136467e-07, 4.758852180046276e-06, 1.1835480680427886e-06, 4.089452013153069e-06, 1.9621332481920115e-06, 4.333353424383096e-06, 2.470753079491589e-06, 1.7354408035902194e-06, 3.162488545921704e-06, 2.5216081638639518e-06, 2.0290385689053928e-06, 2.1524105942260287e-06, 1.8746752216021104e-06, 1.8927290020446059e-06, 1.1677742051252495e-06, 4.275950792158106e-06, 1.7245651289984144e-06, 6.478980735082174e-06, 6.215821618137472e-06, 2.535097418149557e-06, 1.5239906843323277e-06, 2.0636471515782928e-06, 1.737090905842836e-06, 1.8446593407053298e-06, 3.3970537526261704e-06, 3.729361229243489e-06, 4.409360853965529e-06, 1.4167622395025646e-06, 1.648295432426241e-06, 4.409360853965529e-06, 9.865591173178288e-07, 1.6679708869791205e-06, 3.99580143703213e-06, 3.4298136573157553e-06, 2.6695303004112995e-06, 3.835399912025119e-06, 3.2152641549719425e-06, 1.389725323199996e-06, 1.4516419574952615e-06, 7.052251346468665e-07, 2.4274232015092477e-06, 2.8976002955141193e-06, 1.77900682909539e-06, 1.2531353367202255e-06, 9.288427270561168e-07, 2.4963289354825952e-05, 6.7713976487048915e-06, 1.0615644641130052e-05, 1.5847738809161919e-06, 9.052794989087084e-07, 1.2001885010418441e-06, 6.1175100039148035e-06, 1.9355022579885946e-06, 1.0234841130793464e-05, 3.472394477672539e-06, 9.398237348732945e-07, 1.1396547847268563e-06, 5.264007465517188e-07, 2.9895489919261618e-06, 1.0657929637956354e-06, 1.5806045464985836e-06, 3.4465917805258575e-06, 2.3041720675538036e-06, 3.658550225968156e-06, 1.530784794570155e-05, 2.960107317345156e-06, 1.2784342577897674e-06, 1.7610781960927063e-06, 4.220145213023687e-06, 2.50688881717757e-06, 6.26789426596776e-06, 1.1474598461382562e-06, 4.35802732212388e-06, 2.024896311224779e-06, 1.7664644285777257e-06, 2.229510252548993e-06, 2.713650881948908e-06, 1.8040192300483858e-06, 3.346411669616368e-06, 1.5632092118217697e-06, 3.6733485493731297e-06, 3.842867443440442e-06, 3.196424768752945e-06, 3.0017232304936955e-06, 1.3699429170869807e-06, 3.147719124051029e-06, 1.7902428934643717e-06, 1.0778097702335166e-06, 1.8527485911681789e-06, 0, 2.071607460316774e-06, 5.483146687479547e-06, 2.6412861956379234e-06, 1.5062539369614024e-06, 5.255434709298235e-06, 2.9836935874625135e-06, 5.679493080456822e-06, 1.2802077922513807e-06, 3.5199475105733525e-06, 0, 1.924200452231508e-06, 1.918478672132553e-06, 1.2948619018195082e-06, 0, 0, 2.5835704338172546e-06, 1.2966785331929318e-06, 2.0095574727842328e-06, 3.817951165194766e-06, 7.389966267469803e-07, 1.2709889605632684e-06, 4.565661869314121e-06, 2.0177868208665524e-06, 2.969648673520578e-06, 4.839486855918231e-06, 0, 1.0919391682734507e-06, 3.4553519961862824e-06, 2.0942926983212457e-06, 2.0279060181663807e-06, 2.107737060822256e-06, 8.943257097375028e-06, 7.741665047792204e-06, 1.9382647329839805e-06, 6.9169497385157935e-06, 3.997972848973346e-06, 4.3577141197353736e-06, 4.18985339190347e-06, 3.981906335744305e-06, 4.848114511714822e-06, 3.818080312447549e-06, 1.2689852961417936e-06, 2.580577845585533e-06, 4.2435224282239705e-06, 2.6920802049345236e-06, 4.217712467924675e-06, 2.9383825493574735e-06, 2.9745558206411846e-06, 1.022701943818609e-06, 4.489122046201998e-06, 2.522665960130319e-06, 5.337484887425557e-06, 3.8827203462009755e-06, 4.144744987749533e-06, 3.89042671298452e-06, 2.687388421462819e-06, 3.993735406652643e-06, 5.710077166272456e-06, 2.1770354367133664e-06, 4.11731080729763e-06, 0.00012471637747099335, 4.396907184037366e-06, 7.77587927208241e-06, 7.91004201775016e-06, 1.1011271737282105e-06, 2.9446660824777988e-05, 1.8905995041331689e-06, 2.1949153524796974e-06, 3.5423275030798954e-06, 7.250329899798174e-06, 1.794616225319624e-06, 1.5893775302301526e-06, 2.691787405935454e-06, 8.877478032537531e-06, 4.504649306339432e-06, 5.329325079878207e-06, 2.2242916141721403e-06, 1.923115363020747e-06, 9.699814399025315e-07, 4.333353424383096e-06, 3.1288524070314966e-06, 2.4604885650400485e-06, 1.1647019782608193e-06, 1.357955159747795e-06, 7.951586105977707e-06, 1.644154684578678e-06, 1.4473357304757048e-06, 2.625904212943878e-06, 2.0942926983212457e-06, 1.8040192300483858e-06, 1.1353290566224772e-06, 1.850467404732725e-06, 1.3482520425642774e-06, 2.6164705696747377e-06, 1.7365823265050655e-06, 1.3682917405511074e-06, 2.5370386762233894e-06, 1.0262708488691284e-05, 2.5539387790174514e-06, 1.318687301448527e-06, 7.743255069726908e-07, 1.172015068856457e-06, 0, 4.519711391244619e-06, 2.846008999130285e-06, 9.743040374661852e-06, 4.944639562530195e-06, 1.2534174068290921e-06, 4.185722011393663e-06, 1.543245613386338e-06, 6.418113882091746e-06, 9.209645958790432e-07, 1.6348180565225226e-06, 1.5780124360867935e-06, 1.3433381594689322e-06, 2.433955704088146e-06, 1.456442294528124e-06, 2.167420594019222e-06, 1.5084979810532255e-06, 3.7404088625280433e-06, 2.384524427499879e-06, 1.98154556809277e-06, 7.408532795510192e-07, 7.889468469817715e-07, 1.845957996305463e-06, 2.8711493739368013e-06, 3.0073763922098017e-06, 4.017483391230967e-06, 5.8697569694387895e-06, 5.1724884893801265e-06, 3.1096654133113272e-06, 1.5170939790786876e-06, 6.019678653749735e-06, 0, 3.285039782389549e-06, 2.009386769278387e-06, 0, 2.2190730765171062e-06, 2.3621549744306467e-06, 4.615448073967548e-06, 3.4078257395054933e-06, 2.8356095759329514e-06, 2.0031721858698757e-06, 4.088601798878391e-06, 0, 0, 3.1016989546081943e-06, 2.5120156905774175e-06, 6.529501112680199e-07, 6.747011189091397e-07, 1.237820805173495e-06, 3.093089245263373e-06, 2.141721430645982e-06, 8.454609608250482e-06, 1.506250261846421e-06, 6.424602387616762e-07, 2.7484557037993983e-06, 5.694229801274046e-06, 1.2560759183606248e-05, 2.591692295095091e-06, 9.712735518831218e-07, 1.2411745930192405e-06, 4.047568170418162e-06, 2.9060804466011863e-06, 2.096388253633251e-06, 4.969202384365203e-06, 2.241408193611128e-06, 1.2379250808517475e-05, 3.1400482003987534e-06, 4.333353424383096e-06, 2.501244115900168e-06, 2.891028088849985e-05, 2.44610525107609e-06, 1.2575245851974954e-06, 3.5537706201003987e-06, 5.452153237573047e-07, 2.44227993629147e-06, 4.559654098461927e-06, 2.4735278546523498e-06, 2.6396293128821633e-06, 5.4918046373098526e-06, 2.649110541105909e-06, 2.0217765310262343e-06, 3.0365114732121352e-06, 1.6048390176131928e-06, 2.774013045040797e-06, 4.285762327459079e-06, 3.806814231645354e-06, 1.938797135046243e-06, 1.814880238278149e-06, 1.1510590103571475e-06, 4.585282926642688e-06, 4.469086749203598e-06, 4.110827236811393e-06, 6.52533698387039e-05, 8.449185032608634e-07, 2.8981776967132916e-06, 4.527131381362187e-06, 3.346411669616368e-06, 3.346411669616368e-06, 1.8483902750375534e-06, 3.7249954428795833e-06, 4.72431800536711e-06, 3.070997544677659e-05, 2.5640313713565017e-06, 3.073022892734344e-06, 5.826203974542923e-06, 7.461333123212985e-07, 1.4773393672859973e-06, 2.2551231458336352e-06, 9.95813915141929e-07, 1.8821405173637548e-06, 1.0307863043232685e-06, 5.4869450341059826e-05, 3.0304196358424732e-05, 1.10879805501802e-06, 1.292651263921291e-06, 1.1241469311376395e-06, 2.0126715749800923e-06, 3.253053106652619e-06, 2.2788327953504736e-06, 2.2788327953504736e-06, 2.2788327953504736e-06, 8.60712893781081e-07, 2.2788327953504736e-06, 1.5099334974988169e-06, 4.420189174580727e-06, 2.8874872072649284e-06, 2.4231563656444823e-06, 3.0535109222898327e-06, 2.200216651025045e-06, 4.933433387570753e-06, 7.955163508838204e-06, 3.137344642331714e-06, 8.06731489755864e-06, 3.560515036294465e-06, 4.245135080062732e-06, 3.956266713695645e-06, 9.213499222178097e-06, 3.1720582764138975e-06, 2.053256924394536e-06, 9.504242020792138e-06, 3.6367842952335367e-06, 1.9852367353808625e-06, 5.153661897615073e-06, 4.042068311972919e-06, 1.0919391682734507e-06, 6.69355524599638e-06, 5.969729027785098e-06, 4.92244414510758e-06, 3.6738522836416824e-06, 4.186035236072223e-05, 6.645648331663526e-07, 1.9855804927951412e-06, 8.209029569736652e-06, 4.8615677975164895e-06, 5.448322561465446e-06, 7.85543140619712e-06, 2.932589769381211e-06, 2.6153123795138147e-06, 1.4457110235500645e-06, 7.116488990597559e-07, 1.7098429056344974e-06, 1.4947166239923862e-06, 3.182282962725453e-06, 3.753535310714116e-06, 1.6913877935411938e-06, 2.9495125296660494e-06, 1.139741648642714e-06, 1.7311715234705494e-06, 2.8464694233819026e-07, 4.2215139818352684e-06, 1.876316187657133e-06, 1.257853006782013e-06, 3.4772227024912168e-06, 1.399213790187582e-06, 1.8982305088963047e-06, 6.144856609817675e-07, 1.6933607938075226e-06, 1.5411541889403705e-06, 1.0912542764154154e-06, 1.5635959602026079e-06, 6.775716716826754e-07, 2.0803327741379265e-06, 3.573818840545727e-06, 6.555044931307546e-07, 4.305090914235177e-06, 2.1684097987485675e-06, 2.715247363749484e-06, 5.184187471533778e-06, 1.3394287419338712e-06, 3.7608436236365135e-06, 3.2722259548501967e-06, 4.0271494558572984e-06, 5.491804972297162e-06, 6.48666427463471e-06, 1.843681351784872e-05, 5.275683749678796e-06, 4.394639374367129e-06, 9.979398698757232e-06, 3.256781831689059e-06, 1.6472951402204063e-06, 3.0806067443932025e-06, 1.3211693216922702e-05, 3.5111489774513494e-06, 2.832147147578872e-06, 7.894482048378618e-06, 2.045556314277828e-06, 1.8367657332401673e-06, 5.229330721522928e-06, 5.738009828976457e-06, 4.401151261811583e-06, 4.251726546070982e-06, 6.20412126466973e-06, 7.527898690112739e-06, 2.5770889543528297e-06, 1.688943383785857e-06, 6.444295984147714e-06, 5.938199880543492e-06, 5.968939701214383e-06, 1.8122623581645317e-06, 1.3188737320717301e-06, 9.745946502960067e-07, 1.7059696618793663e-06, 3.1745604360509534e-06, 3.151607195095653e-06, 2.2722741624615816e-06, 1.8170008665739346e-06, 1.9219780114619992e-06, 3.5208628787048956e-06, 2.751992781720532e-06, 2.3586387268386954e-06, 1.3536928618597131e-06, 1.0630344504790938e-06, 5.0936811978036965e-06, 5.495177487380727e-06, 3.562808577042355e-06, 7.52522864345454e-06, 2.036326815576981e-06, 1.1699087359444417e-05, 2.2074023526787807e-05, 1.103379726877783e-05, 5.99132879077721e-06, 3.6209952617149502e-06, 1.9958061974419695e-06, 1.6006538736945204e-05, 2.832848055661383e-06, 4.018955596634637e-06, 6.147798260509364e-06, 1.4654225881668002e-06, 3.474824014666445e-06, 3.5511450756517807e-06, 1.7091809599975957e-05, 2.9126050981120205e-06, 5.211909211708822e-06, 1.6684130457356145e-05, 8.05538998418208e-06, 1.6460392779715655e-05, 6.07266361415911e-06, 2.480363180916266e-06, 6.531237066732024e-06, 6.108213695248238e-06, 2.7437660289460904e-06, 5.539592380112231e-06, 5.612537511252032e-06, 9.696793082891038e-06, 3.2508845083384996e-06, 5.527043241125275e-06, 6.891059349935745e-06, 1.802727664819011e-05, 8.349813130670905e-06, 6.502258694587022e-06, 3.1814120534816434e-06, 7.135612287701908e-05, 1.0160543516275536e-05, 2.817112732799907e-06, 2.179849633869248e-06, 2.0082446996687174e-06, 1.0932339893319271e-06, 1.9762909095147948e-05, 1.678942411430228e-06, 3.0053181444916044e-06, 4.675894611183603e-06, 4.589643782747463e-06, 2.856720910265528e-06, 7.91518947203486e-06, 5.137758736716002e-06, 2.9035026631112496e-06, 1.4129912879628271e-06, 1.5887853347451009e-06, 1.5927048818503305e-06, 7.665508247138586e-06, 3.982236129258485e-06, 7.746782690564061e-07, 1.0442791098682531e-05, 1.5287683477501374e-06, 1.4664704969683646e-05, 1.3155908457570404e-05, 1.9949501869124407e-06, 4.360456628621466e-06, 1.0435167249979856e-05, 1.6478496383909823e-06, 9.971395870298156e-07, 2.337739402443819e-06, 2.2078517808149224e-06, 1.4972932151793303e-05, 7.815637287797458e-06, 1.7274965733291512e-06, 2.8966675756413568e-06, 1.1538003765994585e-06, 2.0654681772381895e-06, 8.989467379025864e-06, 2.4466138750900426e-06, 1.6925052951156962e-06, 3.356681224393915e-05, 2.8221224882415627e-06, 6.37392011953062e-06, 1.5267907884094773e-06, 2.3718984030136804e-06, 1.8367908300216107e-06, 3.317412306425234e-06, 3.230439717917339e-06, 4.184633279997466e-06, 3.1345284993341807e-06, 4.333353424383096e-06, 4.055705142177693e-06, 3.929991558361592e-06, 2.9810530957121408e-06, 1.9531634824781443e-06, 1.5367638727305338e-06, 1.2866672373264772e-05, 8.308234190428645e-06, 2.6575523005210175e-06, 3.4850735114451934e-06, 3.727582296114902e-06, 3.5685992863463223e-06, 2.8796286961119577e-06, 3.411936236419257e-06, 2.827551208736248e-06, 1.5345383637726237e-06, 2.947995088205861e-06, 2.8720304302627914e-06, 5.531744237614421e-06, 1.8326281713681119e-06, 8.186996930027684e-06, 2.3111327090318126e-06, 3.7302032459794404e-06, 9.329389866973858e-07, 1.049850381799015e-05, 4.470391076545773e-06, 5.738352319337439e-06, 5.1400339129702365e-06, 9.405071354869534e-06, 1.0346412752321068e-05, 3.438180722562733e-05, 3.912638044461892e-05, 1.0901936916903804e-05, 0, 7.627503333703744e-06, 0, 1.2365196462787476e-06, 4.382553725858285e-06, 1.698951909229421e-05, 7.807529242626386e-06, 3.27476987380651e-06, 1.691482919621111e-06, 5.777094850841831e-06, 2.1587868305314033e-06, 1.808938925562599e-06, 4.442192953688894e-06, 3.268726227076406e-06, 1.2651732371581308e-05, 9.921148823022665e-06, 6.5609343564978435e-06, 1.2601800344195013e-06, 6.751466372104186e-06, 3.264166798525403e-06, 3.1201052803516403e-06, 1.3175690198414602e-06, 1.3741465304631737e-05, 1.1910551744476626e-06, 1.1990228192366716e-05, 1.1733120819251873e-05, 4.034444578692292e-06, 3.6630071522069382e-06, 2.2062473887458143e-06, 7.030208223713122e-06, 2.0198287709512588e-06, 5.944944554826075e-06, 2.1993547677146967e-06, 7.922431304158317e-06, 9.792676436605374e-06, 3.547765810832343e-06, 2.4070734893289144e-06, 1.2969200308040427e-06, 1.4642752143683246e-06, 3.867821401513505e-06, 3.922247910029905e-06, 1.7689508970137793e-06, 2.5375339356233186e-06, 8.365067150423174e-06, 2.435066406166834e-06, 2.0159568931848756e-06, 1.2782056559209615e-06, 1.153388028544002e-06, 5.075450794358464e-06, 1.1962496515621782e-06, 4.831101726794079e-06, 1.3858494582734616e-06, 3.48721050291991e-06, 1.4913306068612393e-06, 4.2558712840399975e-06, 1.2577316319360126e-06, 1.489657740166858e-06, 3.914211490195454e-06, 1.9903506362450103e-06, 3.784596637020635e-06, 3.911189386858165e-06, 2.3841139653168106e-06, 5.843602565961833e-06, 1.166237666931341e-06, 5.0232565455292805e-06, 1.7570888641386645e-06, 1.3488489509504699e-06, 5.301034510701003e-06, 2.133809399669109e-06, 2.5176322560834e-06, 2.7351564615930853e-06, 1.8829009175412092e-06, 6.047568540360974e-06, 2.3539410798624134e-06, 4.278125370271113e-06, 2.054258806810861e-06, 0, 5.444036960144435e-06, 1.1585281129888486e-06, 1.0863760554747985e-06, 2.2788327953504736e-06, 2.2215299027113485e-06, 3.0203221379099534e-06, 5.174663597651745e-06, 2.36049338348016e-06, 3.6330958323475128e-06, 2.0327196294483917e-06, 1.748615691464776e-06, 3.3769731750205408e-06, 3.898913213623554e-06, 9.143965513489963e-07, 1.1192375202524613e-05, 1.1011271737282105e-06, 5.7636711485606455e-06, 3.0964822504975073e-07, 3.761436561685899e-06, 5.186932068041231e-06, 3.7672023586731083e-06, 1.1836699191970256e-06, 3.7622215539990613e-06, 2.6875804248251613e-06, 8.300242753757566e-06, 4.113198268582281e-06, 3.391485758307024e-06, 2.643464244153825e-06, 1.1887415821562022e-06, 4.452176569927518e-06, 5.501224594174058e-06, 2.258097787282525e-06, 2.8325996274638003e-06, 8.794443913918495e-06, 2.9405485897057987e-06, 1.259161291369586e-06, 2.548957959668925e-06, 4.412570465615748e-06, 4.4326571614095555e-06, 9.352396424535624e-07, 2.2273403539567724e-06, 4.692357469192847e-06, 2.7682029397260087e-06, 1.850467404732725e-06, 9.636681708035009e-07, 1.2314401393008909e-06, 1.3102034045482652e-06, 1.0784863391887844e-06, 3.457189225969769e-06, 2.9151544656130454e-07, 1.5840942123655654e-06, 7.678013527951747e-07, 1.3914313063205452e-05, 1.1140934920663203e-06, 1.2197578371668585e-06, 1.3301473834633076e-06, 3.5521367536438108e-06, 1.6823164182907326e-06, 7.1881986953203285e-06, 3.460121074763983e-05, 2.331564944811675e-06, 2.3376028047702793e-06, 8.59290531955305e-06, 6.111918238091502e-06, 2.981936683180544e-06, 3.0701481413500237e-06, 1.7554184001444314e-06, 1.4164251583046726e-06, 7.156938218323852e-07, 1.9072307049554044e-06, 9.397456013360957e-07, 6.139703096013674e-06, 4.704862866855394e-06, 2.627438888359748e-06, 6.435473080978745e-07, 2.371898258333398e-06, 2.9793289585760394e-06, 0, 1.8267452280586463e-05, 6.81384844720137e-06, 6.760098271630452e-06, 9.436891428392112e-06, 3.006249170696297e-06, 4.4248093908388565e-06, 1.0211928418480534e-06, 5.853275957907758e-06, 3.9470246561375515e-06, 1.69362777062083e-06, 1.0874376979362755e-05, 7.547067193488776e-06, 1.0875559258546786e-05, 5.7296266828626465e-06, 4.054885502951129e-06, 1.6670787732094785e-05, 1.2681097977069702e-06, 1.3325604416466992e-06, 2.3500200294765866e-06, 3.4391696906738096e-06, 5.383929534321792e-07, 1.6956151746329751e-06, 2.7036150262889144e-06, 1.7475892426813884e-06, 2.6321445356307572e-05, 2.6312151415890276e-06, 1.4342642220772308e-06, 2.9748806177379334e-06, 1.7457603506321261e-06, 1.7042059736633556e-06, 7.7663983417773e-07, 1.334252853940977e-06, 3.318862239772076e-06, 8.826229534108241e-07, 3.4211970178574074e-06, 2.8674252145595013e-06, 2.19013612539916e-06, 5.752885210785193e-06, 1.0548287743412721e-06, 2.976391163237993e-06, 3.1806680222771608e-06, 4.962048161929916e-06, 2.4103526237741034e-06, 4.14251401119308e-06, 4.470250101421085e-06, 6.211014141652796e-06, 4.752512940924464e-06, 1.7299936681535769e-06, 0, 1.6637564818616085e-06, 3.5485495015564345e-06, 5.000587315339057e-06, 2.030168315715289e-06, 3.546740413557881e-06, 2.6277125195711522e-06, 5.088233455531552e-06, 1.4005443948279838e-05, 4.1956911897300065e-06, 2.022188165659558e-06, 3.7062103932442992e-06, 2.7349480469347816e-06, 1.3156333578121184e-06, 7.801461432360116e-06, 1.2525928948141725e-05, 3.990146777717507e-06, 2.3939497227583696e-06, 1.137200638299766e-05, 4.308630516821425e-06, 2.2960906366666254e-06, 4.091476714742613e-06, 5.828228983715019e-06, 7.951771146410832e-06, 9.841852102178686e-06, 1.8268604427841144e-06, 1.1330034174617407e-06, 2.3261297998668606e-06, 3.967004933523872e-06, 1.3355332067156062e-06, 4.193569714851329e-06, 3.0487358623033598e-06, 1.437221914846957e-06, 1.6653623982670665e-06, 1.4686983132556974e-06, 1.9982717882417798e-06, 2.7936090433268494e-06, 4.241423261687879e-06, 5.53896753553951e-06, 2.714912983040729e-06, 5.422675838888485e-07, 5.449866143839281e-06, 2.394360527352787e-05, 2.470819355186921e-06, 2.954638123163228e-06, 5.634655218130007e-06, 0, 3.6338598050124178e-06, 5.682120422462265e-06, 1.6583718300404939e-06, 4.038975085453663e-06, 6.396971732332608e-06, 5.042575714504968e-06, 4.261437917198621e-06, 3.9500258985309186e-06, 3.216207648058418e-06, 4.43521861941769e-06, 5.200097712559293e-06, 2.6413199891620383e-06, 1.3131780581679887e-06, 6.807848545517933e-06, 1.5694960375888782e-06, 1.6346647440597192e-06, 2.1037597392177974e-06, 8.421171547738277e-06, 5.293391890226774e-06, 4.855217078332579e-06, 4.2837660548553596e-06, 1.1361736616511587e-05, 6.404764175450308e-06, 3.2366074780471296e-06, 2.5486312771903388e-06, 5.4350400610729965e-06, 1.843801969374186e-06, 2.8902959085362884e-06, 2.9730524631378788e-06, 2.5591884906392142e-06, 2.950727116439655e-06, 3.6524387622738404e-06, 3.488820570507665e-06, 2.2791203074749767e-06, 2.6440488897238708e-06, 4.235590732034295e-05, 2.4275482937276515e-06, 5.4099358376859535e-06, 2.4535524686554956e-06, 4.0542409898282665e-06, 1.2497432645289624e-06, 6.411080705420315e-06, 3.4381479974998935e-06, 8.162815650839326e-06, 2.417311019198163e-06, 8.200664997801052e-06, 4.061880266276389e-06, 1.5180314169633123e-05, 1.283593193079954e-06, 6.1327429035424145e-06, 9.559458201853291e-07, 3.315367487674991e-06, 7.755774234707915e-06, 4.440652319581444e-06, 1.2379091859829055e-06, 2.9703650393267196e-06, 2.678197521471624e-06, 8.28125589175525e-06, 1.8639253935868108e-06, 1.7316620930562291e-06, 4.498379572657474e-06, 2.0910380331535978e-06, 2.2648159809988054e-06, 3.208218348728563e-06, 2.1577186354628015e-06, 1.3173253646402042e-06, 9.158402540510618e-07, 7.975413340506292e-06, 1.7260584056426882e-06, 4.682941192578196e-06, 2.915346603906661e-06, 1.1439765340958828e-05, 3.3037817650707567e-06, 1.2892078236702908e-05, 2.471005607320621e-06, 5.383574948347679e-06, 5.560806428895414e-06, 1.0199572658843565e-06, 9.359070931389837e-06, 7.409263446146426e-06, 2.188055698530813e-06, 2.292327263151551e-06, 4.560840678601289e-06, 5.629668865514272e-06, 6.720314808592023e-06, 4.892210476965855e-06, 1.0272237233166723e-05, 1.0050133545116672e-05, 4.1965361018136144e-05, 2.7562021109919745e-06, 2.2029865021711238e-05, 1.1165936150410776e-06, 2.4243862832909676e-06, 1.2446370707259158e-06, 1.4417988396453434e-06, 1.0804676981997422e-06, 1.2173801785219621e-06, 1.3899920776416545e-06, 3.0241152882769065e-06, 1.7478418203259708e-06, 5.639157176966714e-06, 1.320751718923593e-06, 3.3999839107682334e-06, 2.8328770497302492e-06, 0, 1.6991724483642877e-06, 1.1357409062641433e-06, 2.3675140177961798e-06, 1.9598835227429404e-06, 6.509901358249701e-06, 5.561250214240284e-06, 1.8874751279290025e-06, 3.24996047934414e-06, 2.4548914392215757e-06, 4.2509541558748115e-07, 1.4176650891943757e-06, 1.3938610359415643e-06, 5.3653213879319775e-06, 6.227098486603638e-06, 2.041455048858498e-06, 3.3347513294563278e-06, 1.4913306068612393e-06, 1.4913306068612393e-06, 1.6436875517964228e-06, 3.465272009011418e-06, 2.5389655852124864e-06, 2.2762474079162614e-06, 4.347148955627879e-05, 8.868093561435501e-07, 7.895617131638157e-06, 1.669482220402387e-05, 2.8570457006332974e-06, 1.723803505234825e-06, 2.632969139885306e-06, 8.46032510721972e-07, 3.156543569087769e-06, 1.662823998448686e-06, 2.388968146838995e-06, 5.264007465517188e-07, 5.531327703947514e-06, 3.5605152534776086e-06, 8.435864965345412e-07, 1.1932224520238895e-06, 4.386817246640159e-06, 1.6933607938075226e-06, 1.875994894202046e-06, 2.603909762411331e-06, 6.654006923293344e-06, 1.4463373168300784e-06, 2.614667567190298e-06, 1.2997204269309674e-06, 1.5996701256401416e-06, 1.7987309441097186e-06, 3.770366236842875e-06, 3.535316905722155e-06, 3.814194581529472e-06, 3.574705153127534e-06, 1.392880563911312e-06, 2.0146796840860763e-06, 9.507278997422225e-06, 1.406386863647053e-05, 1.6166544358912433e-05, 1.0962153857490089e-05, 2.7884630082529847e-06, 1.019272321947395e-05, 2.3478783094919677e-06, 2.798035799790286e-06, 1.0730514567076193e-05, 4.868505207642256e-06, 9.472648250182974e-06, 1.1111015425693592e-05, 2.521205707823844e-06, 1.0393563060373097e-06, 2.2788327953504736e-06, 1.059988786688589e-06, 1.522919976813559e-06, 2.303341776754109e-06, 2.065897654553009e-06, 6.190288454269543e-06, 2.2788327953504736e-06, 2.2788327953504736e-06, 2.2788327953504736e-06, 1.1151641948321854e-06, 1.6162488908421015e-06, 6.593526128347132e-07, 2.1379276851503575e-06, 6.7205139489977324e-06, 5.0106347634104436e-05, 2.911572575587788e-06, 7.821205059990283e-06, 2.2788327953504736e-06, 1.6766052663383566e-06, 2.2788327953504736e-06, 8.790175687309446e-07, 9.171804364072462e-07, 2.0057943319651616e-06, 1.0679323557314204e-05, 1.5242529932250776e-06, 4.4973704768960986e-06, 1.4109183656608963e-06, 2.869908427763228e-06, 1.911694901120082e-06, 2.988703798542621e-06, 1.248312457908215e-06, 2.830408994626394e-06, 1.9772326829635153e-06, 2.333389585954542e-06, 4.898746363585553e-06, 1.22706515779399e-06, 2.436055096516969e-06, 5.7133073562543986e-05, 3.7872091623849637e-06, 5.745860723147114e-07, 2.0256341945500697e-05, 8.416942522074307e-06, 5.6232547740011726e-05, 3.8586653282791935e-06, 2.7718502638086333e-06, 3.1584820011428654e-06, 1.5760751730650643e-05, 6.30562849012008e-05, 8.040616313209595e-06, 7.948396472518748e-06, 3.1859659465251625e-06, 2.053086411909878e-06, 0, 1.7932047057859473e-06, 2.5660612356878886e-06, 3.2934652255872846e-06, 4.903637606905261e-06, 8.998984957179408e-07, 1.3317332366723796e-06, 4.0009806776178855e-06, 5.535088440424581e-07, 2.621468099809527e-06, 1.5211765083060034e-06, 2.7675730595601126e-06, 1.1931949400668732e-06, 7.951437203696959e-07, 8.974625480407449e-07, 6.56078258174533e-07, 1.3199590177145015e-06, 1.9886901509395367e-06, 9.375952429423363e-06, 7.812112026687332e-06, 1.0458232533406239e-05, 1.8406021256028265e-05, 6.170061165570192e-06, 0, 1.7578653283207074e-06, 2.634986897213256e-06, 4.6994769346923775e-06, 1.2949534667535968e-06, 2.2788327953504736e-06, 1.0860322865955608e-06, 9.25635146658104e-06, 1.3697979006011679e-06, 1.496429811629032e-06, 1.1965608659768718e-06, 1.3008415715555677e-06, 2.1301627427942324e-06, 1.7140595867812297e-06, 1.4928679378729814e-05, 2.581125458302127e-06, 2.9823727992251453e-06, 6.867942580799547e-07, 1.6139840036853356e-06, 1.6604437022374205e-06, 2.0295265156351603e-06, 2.4570382646843114e-06, 2.366932574769995e-06, 1.2370554671391634e-06, 1.0919391682734507e-06, 1.0919391682734507e-06, 1.4939638967374941e-06, 2.120008048387038e-06, 1.5855684937449528e-06, 0, 0, 1.1710361010216072e-05, 0, 3.0473479436508084e-06, 3.5669743766330365e-06, 2.3420417905620038e-06, 1.276119116344846e-06, 1.0919391682734507e-06, 1.5963210659412323e-06, 2.0491877798416564e-06, 1.4266188611110366e-06, 1.0590484145596294e-06, 1.1832931615671515e-06, 6.10477941050488e-07, 1.2435210543843551e-06, 1.2755381707974301e-06, 5.105128133389224e-07, 1.0445577965595569e-06, 1.3484310092003754e-06, 5.817540215865577e-07, 1.8840827767538753e-06, 4.5912843474377957e-07, 4.83105494562312e-07, 5.050192661987638e-07, 1.4318692610424901e-06, 2.9456238383285633e-06, 2.204893734530824e-06, 2.7142409649202163e-06, 1.2298355652503258e-06, 7.116488990597559e-07, 0.00014535020551064245, 1.714483761130067e-06, 1.5198460896888507e-06, 7.116488990597559e-07, 3.7685191164325976e-05, 1.2694504702331195e-06, 1.7224190782401958e-05, 6.776071563929885e-06, 2.3519076533306965e-06, 4.5682609068328725e-06, 6.073406703407038e-07, 1.0499190625948086e-06, 2.3470880382965005e-06, 2.0279060181663807e-06, 1.1168000062418178e-06, 5.4226073698038355e-06, 1.7403476802054835e-06, 2.2464932633535522e-06, 9.193999869217254e-06, 2.2129801137805815e-06, 2.0026578063426044e-06, 6.286007831776065e-07, 8.950324868047984e-06, 2.2788327953504736e-06, 2.2788327953504736e-06, 4.361219508343725e-06, 5.2169065170861126e-06, 8.686066991530637e-06, 5.9638938136104245e-06, 3.963651456819798e-06, 2.3198649349193616e-06, 2.2788327953504736e-06, 2.018962210724582e-06, 4.14711306862925e-06, 1.7523325492823457e-06, 3.6627011708696766e-06, 2.3921179516185733e-06, 3.8866417292389084e-06, 2.3463450838498163e-06, 3.949258390880004e-06, 1.8773155530379094e-06, 1.7590381519620139e-06, 1.5554463870304902e-06, 2.0448638907758017e-05, 5.3119106564734554e-06, 7.6006364238572806e-06, 6.757884708335631e-06, 1.356431945342479e-06, 3.7320370400660547e-06, 3.6219409394499083e-06, 1.3527600359294708e-06, 1.400836613537357e-06, 2.339328897219361e-06, 3.256791077488943e-05, 1.5184217107365658e-06, 1.5254099591544e-06, 4.6767585471089466e-06, 9.992323645659395e-07, 4.048094331985818e-06, 2.5672707384837735e-06, 3.7458447619546585e-06, 1.2068440616517784e-06, 1.0729545896885193e-05, 1.515093652862004e-06, 8.49091050750566e-06, 2.4820856828948495e-06, 6.878851546173398e-06, 2.2506012261147417e-06, 2.5736517840603617e-06, 2.9089711560184225e-06, 5.484090696193966e-06, 2.598900405794801e-06, 5.252204203837875e-06, 6.909768208307955e-06, 0, 2.5484022163201154e-06, 2.2487432028023642e-06, 1.6302993141745e-06, 4.8153031997077195e-06, 1.6389574341681535e-06, 4.8781242688101306e-06, 3.917406298206386e-06, 1.6415456416118654e-06, 4.663237509649849e-06, 7.884808443163777e-06, 1.914556525259104e-06, 2.6217016092314196e-06, 2.9562094812525036e-06, 3.3040755986751022e-06, 7.281660836527392e-06, 5.271778329147464e-06, 6.773877561654619e-06, 1.3385162572983409e-05, 2.4246774802863813e-06, 1.7672465450273373e-06, 3.580755558869798e-06, 4.315763141776969e-06, 1.2138289559078961e-06, 3.1718299766434026e-06, 1.8339185247215685e-06, 4.29465379215227e-06, 2.7014928912402283e-06, 2.4313312083675137e-06, 1.4795626706111248e-06, 2.9370165623638216e-06, 5.804569436173347e-06, 5.547638334150902e-06, 6.8572014923779e-07, 3.474443574146603e-06, 5.5829904950283075e-06, 2.986091605615556e-06, 3.4084123975660842e-06, 4.864087000976183e-06, 3.3848954117864607e-06, 4.442192953688894e-06, 5.015464767397564e-06, 3.268726227076406e-06, 4.0971796918469e-06, 5.2093538070861774e-06, 2.608845652452264e-06, 2.6195051806571707e-06, 5.075450794358464e-06, 2.4454978313020918e-06, 2.2996868014995572e-06, 1.0023535699158974e-05, 1.4266187740906064e-06, 1.4893669530160004e-06, 1.273543298517044e-06, 4.829331356939881e-06, 4.357714385545801e-06, 2.5413282341880725e-06, 1.7744482286286985e-06, 4.5275695057853345e-06, 1.2762153305799038e-06, 2.1811394507431396e-06, 1.8982306825777785e-06, 3.263324279923506e-06, 2.1670772780366896e-06, 2.6208014271574403e-06, 3.1635668244234585e-06, 5.868017145538062e-06, 4.933438051956366e-06, 9.321874044331686e-07, 2.735868411049705e-06, 2.2222947746367047e-06, 6.344447877320778e-07, 4.716918765444415e-05, 0, 4.406100708898291e-06, 6.103136578623055e-06, 1.7874019199648975e-06, 6.239547777285926e-07, 4.937837025063459e-06, 4.625927014964619e-06, 4.117262147973572e-06, 2.73704238293555e-06, 2.996110849948772e-06, 2.8640206753538787e-06, 2.6246479481302378e-06, 5.3485353912752195e-06, 2.870561248544135e-06, 1.7626195655903161e-06, 3.7496842860059464e-06, 4.863028794359998e-06, 4.570382845042612e-06, 1.6080914770279417e-06, 2.368564276501097e-06, 2.5941942438977538e-06, 1.8732278704219687e-06, 3.618150661072592e-06, 2.7080924954715584e-06, 9.403155290479136e-06, 4.3108219117522015e-06, 5.441274559887751e-06, 5.096836428502208e-06, 3.6120385009124038e-06, 2.0041911638895545e-06, 2.876225895960123e-06, 3.7973976621435772e-06, 3.4259663406920236e-06, 4.979513128708978e-06, 4.334481113238245e-06, 2.5563671304331745e-06, 6.674122695164049e-06, 0, 0, 0, 0, 1.5527268895391104e-06, 5.261733179183766e-06, 2.562232972823325e-06, 0, 1.6193582196814717e-06, 1.0201188666477234e-06, 4.559712436244855e-06, 3.1894679594778922e-06, 4.807537429699335e-06, 8.900617290436187e-07, 7.374665615042472e-06, 9.356537987297851e-06, 2.3758958841167604e-06, 2.8858060516171495e-06, 2.182103371026979e-06, 1.8675335897807733e-06, 5.421069523980374e-06, 2.57099356638031e-06, 1.6579478348436668e-06, 1.7963225383165473e-06, 2.680932238480177e-06, 1.4487077453208851e-06, 2.2367253500592347e-06, 9.304232209527859e-07, 1.5620882252701845e-06, 2.2073907731179673e-06, 2.6106987570308226e-06, 1.0819988066751753e-05, 1.0785536723999688e-06, 0, 2.3937140494208247e-06, 0, 1.9390382273577946e-06, 8.627094090338631e-06, 8.382077755510959e-06, 2.0014268375382186e-06, 5.42641516626322e-07, 2.4504439975761824e-06, 1.748361562267954e-06, 2.742615312585681e-06, 2.4894383869018516e-06, 7.510316176153342e-07, 3.5589395041535125e-06, 2.4084487229071266e-06, 4.071157947320872e-06, 1.4840363835376225e-06, 1.9467693874712957e-06, 9.153411882542387e-07, 3.831005682965425e-06, 6.105239779354516e-07, 1.5381238070908468e-06, 3.4846529010899125e-06, 1.4956345744467078e-06, 6.467272346937318e-06, 2.6353365039600402e-06, 2.667654651050523e-06, 1.1221221772757507e-05, 4.282706419409674e-06, 4.4901173771369655e-06, 1.1448778668028198e-05, 8.914092091891475e-06, 2.1661266678534783e-05, 1.146524222589937e-05, 7.650612489090077e-06, 1.992908588920724e-06, 1.3855067591977292e-06, 9.233180448851809e-06, 1.2486655121340769e-06, 2.5871718652812606e-06, 2.390151005984129e-06, 3.2533127609332566e-06, 7.013405887987763e-07, 2.2741873483765748e-06, 2.1282998031260036e-06, 3.1337437816626657e-06, 3.2444998916685764e-06, 3.244018518733827e-06, 3.335367826727215e-06, 1.1438355321302683e-06, 5.1105245256781045e-06, 4.6342126236444245e-06, 6.807684414824849e-06, 3.1364562338619872e-06, 1.1454994641369942e-05, 2.9645676617780496e-06, 1.5903832550182163e-05, 4.724306178305931e-06, 1.7729209736543738e-05, 3.1060380614085683e-06, 9.375952429423363e-06, 1.6331126609696093e-06, 4.5832575678738955e-06, 1.6868827414641242e-06, 1.330358231799881e-06, 3.941131900484175e-06, 9.148130144680471e-07, 1.4708661071498318e-06, 1.6618300121046187e-06, 1.8242437532785113e-06, 2.9951518553686496e-06, 1.0663916831181866e-06, 2.628783918255021e-06, 1.446298013923078e-06, 1.8133101739017416e-06, 1.5993057202064381e-06, 1.330358231799881e-06, 1.2643310819061667e-06, 5.3414249744195816e-06, 3.0264308285734006e-06, 3.4833647762006503e-06, 2.2788327953504736e-06, 2.3781017831178375e-06, 1.8843310016536829e-06, 3.104190315600409e-06, 2.4029369615351123e-06, 1.6064032526970602e-06, 5.751643991323036e-06, 2.2788327953504736e-06, 4.371227644347911e-06, 2.5526230520706956e-06, 2.4101113602156124e-06, 3.6674819330579125e-06, 7.1140409311992725e-06, 1.4363912672308873e-06, 5.824646826844289e-06, 3.164061542254888e-06, 4.397424960003768e-06, 4.504649306339432e-06, 7.055967926851881e-06, 1.7597986781840136e-06, 9.645208798397492e-07, 5.83471396645476e-06, 1.9272588004236162e-06, 8.10310236749598e-06, 1.5404721324124032e-06, 2.2788327953504736e-06, 2.882042620327743e-06, 1.1112795160029001e-06, 2.2788327953504736e-06, 2.4866461806075613e-06, 2.328103703008664e-06, 1.7918450101843903e-06, 2.2765107789177814e-06, 1.4011833607926926e-06, 2.4035568644466968e-06, 1.4215128723263914e-06, 1.4967811390869607e-06, 2.2989549601748467e-06, 2.6845374036455595e-06, 2.4188153270453826e-06, 2.26007396238267e-06, 2.728651056153856e-06, 2.779402525525303e-06, 1.2279763601523064e-06, 1.4779933811161137e-06, 1.714483761130067e-06, 2.852746649987778e-06, 1.748771930861749e-06, 2.2097841852917606e-06, 4.0406670030007886e-06, 3.874281193125137e-06, 6.853945798400799e-06, 8.367933309608784e-06, 9.773783326703494e-07, 1.7389508368699578e-06, 1.5761609976310441e-06, 6.127662700920816e-07, 1.061539975634219e-06, 7.4702769054036805e-06, 9.170480220845289e-07, 7.5174585748556976e-06, 1.186880645091363e-06, 5.116767770085761e-06, 3.848418915928019e-06, 2.7134296718398266e-05, 1.452974622369424e-06, 1.8496678919594096e-06, 1.1254354887875382e-06, 4.394639374367129e-06, 1.138232847847217e-06, 2.6875804248251613e-06, 2.3336299603806797e-06, 2.160860424472783e-05, 7.461750483561794e-07, 1.1740517332683833e-06, 1.1965456846878126e-06, 5.098570041546371e-06, 5.476578751971445e-06, 1.703301955839048e-06, 2.4035568644466968e-06, 2.8477591205047342e-06, 3.911189386858165e-06, 2.08011868078962e-06, 2.7769165712183775e-06, 1.8725044140412702e-06, 2.488287326765999e-06, 2.7769165712183775e-06, 1.8725044140412702e-06, 1.1175862427844285e-06, 1.3770795333187964e-06, 6.654604099487199e-06, 3.6630071522069382e-06, 2.7946785309198235e-06, 1.6823164182907326e-06, 1.1064731050200366e-06, 2.4735278546523498e-06, 1.6823164182907326e-06, 2.6363357106552874e-06, 4.88149605354461e-06, 1.8684966160845216e-06, 1.516161493731946e-06, 2.6917873648872582e-06, 3.5119703692868964e-06, 5.7234787889846504e-06, 2.447640697024478e-06, 4.876306476828733e-06, 1.2715518175742691e-06, 1.7971989679185393e-06, 2.497144222038797e-06, 1.2681098363828425e-06, 2.271682091070336e-06, 5.383980683982839e-06, 1.3707780736869082e-06, 5.790216322899095e-06, 5.53896753553951e-06, 1.9589930899742417e-06, 2.078813914351986e-06, 2.9289391040384388e-06, 1.4070422701575774e-06, 1.2207929715964398e-06, 3.898913213623554e-06, 1.0671414553821803e-06, 1.6709159488125797e-06, 1.5082397864284463e-06, 5.163400257453567e-06, 5.972180372422268e-06, 7.556798584136066e-07, 1.3052893108561322e-06, 7.556798584136066e-07, 1.5576116468871095e-06, 3.9084910979747225e-06, 2.830408994626394e-06, 2.3507639324834708e-06, 3.3925580457731925e-06, 2.440175142084099e-06, 8.073861823580888e-06, 2.2095057905828987e-06, 1.2035699706754532e-06, 7.193264951345891e-06, 5.33682157128823e-06, 1.8725044140412702e-06, 2.7769165712183775e-06, 0, 1.8725044140412702e-06, 2.9673359426791393e-06, 1.9910157832372817e-06, 2.326735174999536e-06, 1.759062025758982e-06, 4.3333055158360015e-06, 1.4835624405326548e-06, 2.241408193611128e-06, 2.9104181436079807e-06, 1.9245436851786455e-06, 1.9903506362450103e-06, 1.9903506362450103e-06, 1.2228505666614974e-06, 3.6362092879238384e-06, 3.0042781699934816e-06, 1.6240958674253636e-06, 2.071831509184856e-06, 0.00013211959160842814, 5.238985488032701e-06, 3.916353733639973e-06, 3.0637207780918452e-06, 9.756890481770861e-07, 2.8985595270008283e-06, 8.437096416780988e-06, 1.0199465182961368e-06, 1.3162771248740917e-06, 5.1753775511957155e-06, 6.066605004456272e-06, 4.797858106399286e-06, 5.257234533147281e-06, 5.2511795131057295e-06, 5.478185558796553e-06, 2.564841494651333e-06, 1.5262153957800554e-06, 4.6138266136840773e-05, 1.08258225021315e-06, 2.3277972324506364e-06, 2.0146796840860763e-06, 2.2956953621029164e-06, 2.2197999680749033e-06, 1.9192765332309256e-06, 2.661670930447688e-06, 1.4737389657781818e-06, 1.6410610816883036e-05, 5.248142173934897e-06, 2.700262805795739e-06, 4.041962581596276e-06, 3.285313061688689e-06, 3.7321232049122906e-06, 4.36727922420666e-06, 6.697293992477272e-06, 2.8669846157965015e-06, 2.175549811896703e-06, 2.175549811896703e-06, 2.30836610059079e-06, 1.682316700488575e-06, 2.2788327953504736e-06, 1.7918450101843903e-06, 1.0050983285543092e-06, 2.1011016470071814e-06, 1.174921382698833e-06, 1.1596453041273598e-06, 2.77770343575707e-06, 5.096133387574147e-06, 4.782076771267015e-06, 2.166540897861023e-06, 4.01947215443814e-06, 4.524891575418414e-06, 3.5527171572219702e-06, 1.7764033655043921e-06, 2.66390539569345e-06, 2.380951934422972e-06, 4.954898572565193e-06, 1.5338592351585378e-06, 3.863002212689832e-06, 1.5762058005119053e-06, 2.719901062134228e-06, 2.5950677955996346e-06, 4.247281032800089e-06, 5.599290889907914e-06, 2.6000508917770326e-06, 6.024433115668516e-06, 4.939507291891054e-06, 5.59212725634596e-06, 2.121441413788212e-06, 4.868505207642256e-06, 1.9819308171779162e-06, 3.055385988854903e-06, 1.0207073725590876e-06, 1.580624648778797e-06, 1.1011271737282105e-06, 3.93070994602354e-06, 5.159137600491198e-06, 1.9018988436457486e-06, 4.477251187913795e-06, 3.430563925188632e-06, 1.8824110488984375e-06, 4.0508853535315345e-06, 2.0916677709151064e-06, 1.8137120008980102e-06, 3.1146927357372177e-06, 1.460680589878558e-06, 1.176265880644014e-06, 7.359384858276215e-06, 4.113621927496776e-06, 2.688742614515498e-06, 1.8040192300483858e-06, 1.6837890062379165e-06, 1.222673574932884e-06, 4.8177427527821745e-06, 0.00012537631546308506, 2.2703723493789446e-06, 2.2093528939603512e-06, 3.213438566278378e-06, 1.2408729334165073e-05, 2.290441917459543e-06, 4.168098842700379e-06, 3.2840453478689386e-06, 2.822544248359673e-06, 2.3841139653168106e-06, 1.99827160540689e-06, 3.897146576930385e-06, 1.174401532315524e-06, 5.11845750550653e-06, 2.4016851097656766e-06, 4.218756022442096e-06, 4.0339099183053175e-06, 7.006337680296909e-07, 3.577541237741534e-06, 1.4465822452032739e-05, 5.7145222784378896e-06, 2.2572968410105725e-06, 2.789697271895473e-06, 5.901945926305602e-06, 2.7645959121672464e-06, 4.740399025080631e-06, 2.262348439678178e-06, 5.258881397936434e-06, 3.213843651181422e-06, 2.5622327774605187e-06, 2.649795163419203e-06, 6.305940632025359e-06, 6.402718647381683e-06, 5.502217443651364e-06, 5.324731698810091e-06, 7.216545855772982e-07, 1.780380837308617e-05, 3.947948858795153e-06, 3.52611197971595e-06, 7.883954674070764e-06, 3.9881893807514895e-06, 1.4417988836185322e-06, 1.6049611907895157e-06, 2.68050725605864e-06, 1.376664222641089e-06, 2.974337872641405e-06, 1.8956321036752788e-06, 2.9503758805083387e-06, 3.0701483286219263e-06, 2.0279060181663807e-06, 3.1806681677871754e-06, 1.1309672131178075e-06, 2.8582328271697303e-06, 1.0284511768501294e-06, 2.0450520392211526e-06, 3.239975729558173e-07, 8.184165142145081e-07, 1.4737389657781818e-06, 3.2090713937938742e-06, 3.4506218519548346e-06, 1.616233560570536e-06, 2.555760936956861e-06, 1.3967951140843365e-05, 2.0234325207511033e-05, 2.317861433650229e-06, 2.0189624570285964e-06, 1.467106182390496e-06, 1.7348885781242925e-06, 2.6105786878919595e-06, 1.795060553821908e-06, 2.38100712342396e-06, 7.711143613045833e-07, 2.5716298826244643e-06, 4.303980913608127e-07, 4.223228738956084e-07, 2.155068271617669e-06, 3.877541564715346e-06, 1.034474692143989e-06, 2.4375509282244185e-06, 2.5592645927365414e-06, 4.394858588172691e-06, 2.6907823233866863e-06, 1.2307646684706872e-05, 6.1777771036727026e-06, 0, 3.0264308285734006e-06, 2.382061333634907e-06, 1.2313094461641517e-06, 2.904748656104833e-06, 3.533461703990466e-06, 4.5319169385236205e-06, 2.569594522610426e-06, 2.178692839657381e-06, 1.2587372789450313e-06, 7.102750754219727e-06, 1.105132188426534e-05, 2.0626194589842777e-06, 3.891815681321123e-06, 4.586134736577704e-06, 1.978963360645807e-06, 2.643143264820016e-06, 2.4397240345678457e-06, 2.3611321856134796e-06, 1.1082564043834297e-06, 2.2609395395644752e-06, 7.058922364157114e-07, 2.408201706796872e-06, 2.132942239477734e-06, 2.8872139544949007e-06, 5.998653553466516e-06, 3.6955450812289293e-06, 4.34699390258933e-06, 1.9272588004236162e-06, 4.156308316450174e-06, 1.1159083362093069e-06, 5.421799781000201e-06, 3.60582135714583e-06, 1.1058452000389173e-06, 2.6123811016856974e-06, 2.1504418191750855e-06, 5.05024541582665e-06, 2.4214207368019844e-06, 2.6141844808608615e-06, 2.735833324416299e-06, 4.646655378289563e-06, 5.09581438540329e-06, 2.5216078562395026e-06, 2.657387864092912e-06, 1.5191190924888356e-06, 1.2587202338901795e-06, 1.1584745835790093e-06, 4.674814184999246e-06, 1.7996229564022646e-06, 3.403924394098956e-06, 9.233993045059104e-06, 6.360574360992136e-06, 1.0034237302892932e-06, 2.5875818919580833e-06, 5.072331311409905e-06, 3.6632835516258636e-06, 2.773204473929762e-06, 1.2718502712480267e-06, 1.966056358491979e-06, 2.381826975940484e-06, 9.053123277605124e-06, 4.010366932001298e-06, 3.5119732612854076e-06, 3.8028300816582378e-06, 4.583195504186777e-06, 0, 4.975241421521715e-06, 4.328447697746025e-06, 1.3025555321628357e-05, 3.6939211768051506e-06, 9.839113627300957e-06, 1.4588345807328243e-06, 3.688684442099631e-06, 1.9096585949693403e-06, 1.5996701256401416e-06, 1.754641625413229e-06, 3.63827583391182e-06, 1.3280492632329579e-06, 2.1939650101560307e-06, 9.100595990007141e-07, 3.2778679646510387e-06, 2.0020614613114148e-06, 4.415616484351415e-06, 1.7125937966902627e-06, 6.069367770068629e-06, 7.649488714648196e-07, 1.3282007769301662e-06, 7.838707974779053e-07, 4.682886622126749e-06, 1.7583001822964213e-06, 3.6324771408947845e-06, 1.927054955308926e-06, 3.1282416425234067e-06, 2.80568758834022e-06, 0, 3.80611065197484e-06, 3.238328597689962e-06, 2.1912996972510476e-06, 2.2208129092743155e-06, 1.967236833377704e-06, 7.3471888082666746e-06, 2.5941589566590436e-06, 2.8916400337751082e-06, 4.086319597101981e-06, 3.4259662362040352e-06, 4.530989808121848e-06, 4.853655767873846e-06, 3.818205718231907e-06, 0, 5.015464767397564e-06, 2.833100134707488e-06, 2.3550460815960224e-06, 4.086089104125417e-06, 2.851481784159503e-06, 4.09248941021607e-06, 4.458149043740787e-06, 1.597422965374786e-06, 1.2654538872380045e-05, 1.4132345563204468e-06, 4.1302782833717875e-06, 3.985974827375724e-06, 1.1445379261340275e-06, 1.5618421505531682e-06, 2.55749890470843e-06, 1.6846227393881223e-06, 1.1881995208169898e-06, 7.186089776555346e-06, 5.454636594338324e-06, 3.8018015773415104e-06, 9.626376967942341e-06, 4.564594106055927e-06, 2.2494591971026836e-06, 2.23518001255798e-06, 1.1043200725592693e-06, 8.560793705539407e-06, 3.0824271419183375e-06, 3.1977768126628726e-06, 3.6854229740819907e-06, 2.1722282937920776e-06, 2.4894845497150194e-05, 3.5267970908082732e-06, 5.57540166040228e-06, 3.546283905301895e-06, 5.165788171427105e-06, 2.8102523823721395e-05, 1.9084294070196226e-06, 6.037971712831626e-06, 4.359510637447678e-06, 1.4953973946562811e-06, 4.2015769738146025e-06, 2.7485976226847117e-06, 3.0302835903642123e-06, 1.7634000334773127e-06, 1.3460549979099507e-05, 1.0361189623196485e-05, 1.599075509504397e-05, 8.386185548718145e-06, 4.823546293729455e-06, 7.239975638166716e-06, 1.4408944630089705e-06, 7.469790702867352e-07, 1.8285016586922847e-06, 1.8285016586922847e-06, 1.364807114148448e-05, 6.429845214502064e-06, 9.082747964996275e-07, 1.8567678521927811e-06, 5.1719197354198485e-06, 3.1543257854881856e-06, 1.0741617972907458e-06, 6.118320547836391e-06, 1.2814962173671342e-06, 0, 3.6404971069028715e-06, 1.276119116344846e-06, 5.038670371944278e-05, 1.3624091240609857e-06, 4.819727954859393e-06, 1.024394127292815e-05, 2.373795630567298e-06, 1.6615898132033916e-06, 1.446337581499741e-06, 3.83746256442816e-06, 8.437305621858574e-06, 4.838491294262985e-06, 0, 6.523759581153984e-06, 1.1675621325623218e-06, 4.118525655966933e-06, 1.7731015113227016e-06, 3.67539383960206e-05, 2.662726665141813e-06, 8.929610706543565e-06, 6.087047344203689e-06, 4.127079424798497e-06, 5.176630186940022e-06, 5.354259554526209e-06, 6.54043021106354e-06, 4.920926301547298e-06, 3.857569191199206e-06, 8.830025656309081e-07, 1.0386501144589675e-05, 4.534175929079976e-06, 6.008125677633888e-06, 1.8446844328201856e-06, 5.78522604945946e-06, 3.1575040255075264e-06, 0, 1.4120737143272172e-05, 1.6595611018466685e-06, 1.0390129023970303e-06, 2.8734937910492958e-06, 8.367933309608784e-06, 4.200137862388639e-06, 3.528591271112196e-06, 9.954099087502946e-07, 7.619518324209321e-06, 7.98118838664612e-06, 5.685501809113529e-06, 2.4148821785993392e-06, 7.0799977172772475e-06, 8.050067754632953e-06, 9.903479201333825e-06, 5.754993792388875e-06, 1.8231902590653898e-06, 1.942692102360656e-06, 5.845568608103796e-06, 2.965984899931906e-06, 3.0766529828917063e-06, 1.2197420856623862e-05, 5.784716240839211e-06, 1.9475039898650175e-05, 3.5379139237888243e-06, 7.439855845222825e-07, 1.4913306068612393e-06, 1.4913306068612393e-06, 6.9646645447505005e-06, 2.2762474079162614e-06, 2.0126526073694927e-06, 3.852322628543384e-06, 1.1733743667685046e-06, 1.4145501243991633e-06, 1.2999275225870304e-06, 8.820140141137337e-06, 4.556328271797805e-06, 2.818333042030099e-06, 1.2484439657928704e-06, 3.3417556946046103e-06, 1.7927201026667592e-06, 1.821619561384439e-06, 8.574845394490448e-06, 2.564668623928593e-06, 1.0693700625015097e-06, 1.8959565868360797e-06, 1.7828872478711767e-06, 2.2762474079162614e-06, 1.248865291682515e-06, 2.9138946776368627e-06, 1.4783590690764235e-06, 1.180262752209119e-06, 2.121375354589425e-06, 1.3759643150896702e-06, 2.983214879159817e-06, 2.3095384939252635e-06, 3.8023164905216426e-06, 2.9561692246619265e-06, 1.1616782941023017e-06, 1.3799368095532014e-06, 0, 1.7159831455073062e-06, 1.0524392456219043e-06, 2.224407654623734e-06, 4.9536725475951196e-06, 3.3926929721298557e-06, 2.587872248446483e-06, 5.300788931475983e-06, 7.941683271515782e-06, 1.898028296544217e-06, 7.784038816094725e-07, 2.735833324416299e-06, 5.301034510701003e-06, 1.975640081437552e-06, 2.092593778049022e-06, 1.3936401655566321e-06, 2.665962404280764e-06, 3.911564263363973e-06, 3.1261561525270163e-06, 1.9711904379041583e-06, 9.218528123702608e-07, 1.9852846895100258e-06, 5.116767770085761e-06, 0, 2.7904648218189436e-06, 3.1023726300840987e-06, 1.9262972077393473e-06, 1.4857715191353181e-06, 1.8065569901859697e-06, 4.220091670202613e-06, 7.204096187571972e-06, 2.112026817438404e-06, 2.6542659361458157e-06, 5.35019413125185e-06, 2.4048923212811006e-06, 2.231461068972853e-06, 3.1570869688593433e-06, 1.8593829058093426e-06, 3.3917414626560727e-06, 3.82172135612496e-06, 2.933061598926212e-06, 2.192994077702326e-06, 1.1295588745280681e-05, 4.640519164831807e-05, 3.6933539710530276e-05, 4.9740758226207304e-06, 3.634045142694263e-06, 5.93147696867553e-06, 4.3845381318070645e-06, 1.8034603634896002e-06, 8.742855182691862e-06, 2.893044349115293e-06, 3.992872881857641e-06, 2.3938906653533635e-06, 3.213045830677437e-06, 5.201727802594333e-06, 2.6201535433936785e-06, 3.0356486089091873e-06, 1.2275305580830598e-06, 1.7581917268326031e-06, 2.06365492191698e-05, 7.319989700099946e-07, 4.417185401547966e-06, 2.4491851803082364e-06, 3.559771696027203e-07, 5.7783777932834535e-06, 9.60229239658371e-06, 4.954168433671656e-06, 7.931166992454696e-06, 4.965825979603168e-06, 0, 1.6320971170405237e-05, 4.837981718371026e-06, 6.086340561936051e-05, 2.574218254667754e-06, 9.360853393329089e-06, 7.856405824712636e-07, 1.5920003022612406e-06, 2.3276024300247512e-06, 2.6951820957357697e-06, 2.078869232815382e-06, 8.862643172529364e-06, 4.063376052659431e-06, 7.52225408401168e-06, 2.060863137928843e-06, 1.5181398473448823e-06, 3.193432191182413e-06, 1.469367323473532e-06, 2.1216012972091245e-06, 1.7777795650050496e-06, 1.9037785185987565e-06, 5.697072924938713e-06, 1.983843665153022e-06, 1.9552449264506805e-06, 6.353649097646909e-06, 2.402962245608176e-06, 8.86829790126073e-06, 1.1974849332703883e-05, 4.679923633983838e-06, 7.545829902957903e-06, 2.2103313051068877e-06, 2.0761603268200503e-06, 1.4845803439926577e-06, 2.3970507893609055e-06, 2.601878968715823e-06, 2.0752631113046434e-06, 1.3378369073599639e-06, 3.4365787660653538e-06, 3.2913179618693353e-06, 6.370056772665935e-06, 4.570064834715659e-06, 2.4717841145834147e-06, 2.036830802770143e-06, 5.084855083550434e-06, 2.844532827896244e-06, 1.4637449884342848e-06, 9.897069146751617e-06, 1.7598366245649544e-06, 1.7598366245649544e-06, 1.7456288703642025e-06, 1.891786253291803e-06, 2.713650881948908e-06, 1.059048349960141e-06, 3.547516745905008e-06, 2.9655146404068185e-06, 4.842023992459481e-07, 2.4876893864481234e-06, 3.6929729616765105e-06, 3.233895593193738e-06, 2.045556064729487e-06, 5.528414195233933e-07, 2.5930732384046284e-06, 5.053524859559549e-06, 6.2405176161037134e-06, 1.0642527954256994e-06, 1.8012564032172732e-06, 7.027765499399982e-06, 1.6620221404672566e-06, 2.6281670451563126e-06, 1.407640112980925e-05, 2.9811218310058287e-06, 3.793941002851193e-06, 7.998426494543246e-06, 2.8687586600895775e-06, 4.060337456736592e-06, 4.696652252068439e-06, 3.269535230888465e-06, 8.956028280087167e-06, 1.1258389079627958e-06, 2.6485577873585887e-06, 6.71668089664516e-06, 6.267979174991056e-06, 3.5644656157664384e-06, 1.866844696006582e-06, 3.29565729507183e-06, 6.521369628534803e-07, 8.576734778027033e-06, 2.787719719419502e-06, 3.1501726370411697e-06, 3.5616647722229914e-06, 1.7975626032769848e-06, 6.305940632025359e-06, 7.952179317662832e-07, 1.8858499396553344e-06, 1.1168000062418178e-06, 2.028462022089133e-06, 3.5951711853136346e-06, 2.036830616407238e-06, 1.7140595867812297e-06, 1.9067163337216606e-06, 3.468341454448076e-05, 4.315988293506514e-06, 3.4112240194680997e-06, 1.8530218163883286e-06, 2.8097011890964092e-06, 3.346411669616368e-06, 2.6623313537336585e-06, 1.6462589527208746e-06, 1.2110793511037688e-06, 1.9939957174365065e-06, 3.346411669616368e-06, 3.346411669616368e-06, 6.754293207141355e-07, 3.4693514955998972e-06, 2.3611623947508203e-06, 1.1607480312477843e-06, 1.9409311659025634e-06, 1.1783281521285324e-06, 3.6323023242755465e-06, 3.631818465112929e-06, 2.9917607679838414e-06, 4.084138005775243e-06, 3.4412813684439886e-06, 6.556750681361312e-07, 2.626126223560599e-06, 1.7272499647880254e-06, 1.68115590823566e-06, 1.0966030102276313e-06, 1.4512642218003213e-06, 2.2642554437351097e-06, 6.057117810965386e-06, 3.737239404764528e-06, 2.9504465180348996e-06, 3.142258772669986e-06, 2.68608758613494e-06, 7.16311149738264e-06, 2.994773787176703e-06, 3.643203269744614e-06, 3.959048215403614e-06, 1.0504957646160077e-06, 1.4794818991720073e-06, 2.863738900374887e-06, 2.088320212739806e-06, 3.3234720765755203e-06, 2.481545919710808e-06, 6.073193690525338e-06, 2.93090500669932e-06, 1.138759384490466e-06, 1.1185341075033353e-06, 3.0029830268537003e-06, 2.654468202484567e-06, 2.216279349380737e-06, 1.8659211283293692e-06, 2.6148374281090485e-06, 6.740403324438901e-06, 9.517955405664423e-07, 1.2784342382943897e-06, 2.279759407263972e-06, 2.4448202473358534e-06, 5.741555062346741e-06, 3.160983333274391e-06, 1.7483909968085847e-06, 3.936584777640878e-06, 3.463505165491972e-06, 3.346411669616368e-06, 1.378092383324074e-06, 1.756954869536329e-06, 1.452974622369424e-06, 1.3582017733184481e-06, 1.9957714411071277e-06, 4.998306331003036e-06, 1.880421678122449e-06, 2.6623313537336585e-06, 5.039667872005989e-06, 2.6623313537336585e-06, 2.6623313537336585e-06, 2.350418922892124e-06, 4.954748665138115e-06, 1.8012564032172732e-06, 1.5156165239971918e-06, 4.366261917904836e-06, 2.8125212907241896e-06, 2.0922532213868157e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 4.051339106373165e-06, 2.929819397263732e-06, 1.7215158473522627e-06, 3.548396147731427e-06, 2.8368353452289127e-06, 3.1593065974717652e-06, 2.6245227149112876e-06, 2.92615688263351e-06, 2.674691580212292e-06, 5.855540987518335e-06, 7.037566103847054e-06, 1.668207836292441e-06, 5.672818068957234e-06, 3.085429181157848e-06, 2.7033578685238597e-06, 2.3539410798624134e-06, 2.9655914742152855e-06, 1.4862617421535156e-06, 3.1814555229353304e-06, 7.91004274149143e-06, 1.3023191250878875e-05, 3.528983022122416e-06, 3.4553879850839235e-06, 6.27346479058002e-06, 2.2979647927020637e-06, 1.4438086265169146e-06, 3.3561002972616765e-06, 4.0481563722012484e-06, 2.9484294699600653e-06, 6.450001595771737e-07, 2.2536575369446165e-06, 1.4443043197060456e-06, 2.7664978292175093e-06, 9.776553860611035e-06, 3.3596245119714986e-06, 2.263224344456012e-06, 1.7260982830046185e-06, 5.575907987217755e-06, 7.682526694616071e-06, 3.346411669616368e-06, 3.346411669616368e-06, 3.1984315883341295e-06, 1.7813507012045095e-06, 2.8969123016071958e-06, 1.7701720123742627e-06, 6.266427863129463e-06, 3.799130320395968e-06, 7.035567897058925e-06, 1.7052362993354804e-06, 4.452130934680149e-06, 9.078608114682393e-07, 1.4783555747425507e-06, 2.93854897330431e-06, 4.6916207893045645e-07, 2.7131385709563187e-06, 5.985234027633983e-06, 1.5940495372909991e-06, 9.580440018778554e-07, 3.5182596615660416e-06, 3.637434941148336e-06, 2.3175347178937755e-06, 3.547193843204833e-06, 5.354748561172753e-06, 8.242273914229205e-07, 2.4046518242872744e-06, 4.262492745354377e-06, 1.7194552928628786e-06, 4.199955709105458e-06, 1.7585003039848335e-06, 5.059107646169092e-06, 2.5853519137278273e-06, 2.2172886891178035e-06, 1.7075915859652574e-06, 5.169318824428168e-06, 2.898140926270096e-06, 1.2668667459446027e-06, 9.443427073168767e-07, 1.608605202427064e-06, 4.127079424798497e-06, 3.2304806551706374e-06, 2.959529266898197e-06, 5.698224509722667e-06, 1.1871914859320754e-06, 2.036830616407238e-06, 6.324525293206668e-07, 5.474102267805115e-07, 1.5565396778039155e-06, 4.514788195788573e-06, 5.428341258469372e-06, 5.535088440424581e-07, 2.7030167974072683e-06, 3.2620091219553577e-06, 8.718089894926393e-06, 2.0799068931237427e-06, 3.658328510893937e-06, 6.128898332705227e-06, 8.227969258294286e-06, 9.390853945232907e-07, 2.695479758080134e-06, 1.8211030352792696e-06, 3.576653022059224e-06, 1.011683142028426e-06, 2.591692453182294e-06, 7.390751551842776e-06, 2.989358637803621e-06, 9.225106248957732e-06, 1.166453109826147e-06, 6.10604972177452e-06, 2.18929027638125e-06, 2.0279060181663807e-06, 2.668869800685475e-06, 1.850467404732725e-06, 2.3500200294765866e-06, 1.850467404732725e-06, 2.1373695357897915e-06, 1.5667583313946124e-06, 3.346411669616368e-06, 3.346411669616368e-06, 3.346411669616368e-06, 3.346411669616368e-06, 3.346411669616368e-06, 8.780633733530302e-06, 2.6623313537336585e-06, 2.8449862453622254e-06, 1.4767485611591908e-06, 1.2802356835921922e-05, 2.9113161337021543e-06, 3.176323330982977e-06, 2.595984989144963e-06, 1.5028416465759122e-06, 2.042736678529699e-06, 5.528414195233933e-07, 1.0782842332298494e-06, 2.980613725823276e-06, 1.716834233440125e-06, 2.3987359555022267e-06, 4.995673783061456e-06, 3.3206524017804187e-06, 1.8832248296643923e-06, 1.588623233116011e-06, 1.2844968142450939e-06, 1.6279142193361657e-06, 2.6875804248251613e-06, 2.584234650374021e-06, 4.017483513759541e-06, 1.8866849725497428e-06, 3.1274675999653287e-06, 1.0245702869069556e-06, 1.525075679858116e-05, 2.4822410256469034e-06, 2.8311454356941846e-06, 1.8575723318014021e-06, 3.192300407150719e-06, 1.358909803843661e-06, 1.570081665796334e-06, 1.9860716780739416e-06, 2.471039106311071e-06, 2.089790280158887e-06, 2.5979218451135944e-06, 1.268180305785406e-06, 1.0781013016217637e-06, 9.051805501775124e-07, 1.2866055669601916e-06, 8.890427283354848e-07, 2.925084941642888e-06, 2.3498407824454545e-06, 8.610741010133309e-07, 2.101414442403677e-06, 1.9597374998902886e-06, 1.6111619150246198e-06, 3.3090126106843397e-06, 3.450211282704052e-06, 2.281650712338224e-06, 2.51484002703051e-06, 1.987468010061774e-06, 1.3409769500523913e-06, 4.777633177974797e-06, 9.509608974155774e-06, 6.9602245042759725e-06, 3.483608549139415e-06, 8.503905510113375e-06, 2.4677538846190616e-06, 1.4745209419810228e-06, 1.4276027196395028e-05, 2.7673644114436764e-06, 4.393632712894721e-06, 4.1256419752229554e-06, 4.095164512134123e-06, 8.050067754632953e-06, 1.3955829496233785e-06, 4.551292407762558e-06, 3.696540217543537e-06, 3.0936182460653573e-06, 4.1484788577492015e-06, 3.873536790823329e-06, 5.602805110950756e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 1.5362241245127373e-05, 2.2262938126788386e-06, 2.3930004905840115e-06, 1.8240223580275074e-06, 2.3930004905840115e-06, 1.0939455092842992e-06, 6.9836577064353074e-06, 3.603371653323356e-06, 1.6238387122676839e-06, 4.040760539937456e-06, 6.702719674991692e-06, 1.618160546148191e-06, 2.9595293571603995e-06, 1.0419164454263158e-05, 5.462170334109973e-06, 2.1277530983365174e-06, 2.367337660174037e-06, 2.5448248330804444e-06, 2.196458447578065e-06, 2.0386136468649896e-06, 1.234558460758943e-06, 4.788974215151716e-06, 2.9065048479574593e-06, 3.346411669616368e-06, 3.346411669616368e-06, 2.5941589962184625e-06, 1.6351289136214011e-06, 2.713650881948908e-06, 1.9484485605580768e-06, 2.141721430645982e-06, 3.479885395686902e-06, 3.2781621429916468e-06, 2.8663859349913845e-06, 2.373866798874042e-06, 6.279101898425298e-06, 1.7009295526444115e-05, 2.9502025782766096e-06, 2.4874226360886653e-06, 3.346411669616368e-06, 8.921290429321352e-07, 1.4926539549572508e-05, 3.7374470847465966e-06, 1.6706045552027041e-06, 2.9662041808716693e-06, 3.853700764707685e-06, 4.352721805620451e-06, 4.68468254398986e-06, 1.6228200015890904e-06, 3.346411669616368e-06, 2.059849364921023e-06, 3.346411669616368e-06, 1.4454083601902563e-06, 3.346411669616368e-06, 1.7151107253906973e-06, 6.562400759454776e-07, 1.476568100457895e-06, 2.318709752403993e-06, 1.1049899108878634e-06, 1.1598291960706215e-06, 1.4437431487240288e-06, 2.19738034853721e-06, 1.7735788097902705e-06, 2.7188505719749517e-06, 2.806237487857937e-06, 2.186602866442169e-06, 3.299267260799275e-06, 9.88297378518235e-07, 1.1368929523798379e-06, 7.700349991100759e-07, 2.105920773466687e-06, 3.5406111501516316e-06, 3.481208269752858e-06, 6.287708677980983e-06, 4.177230938725918e-06, 5.000703378392123e-06, 1.6078886032539114e-05, 7.2323965957736045e-06, 7.938245774497546e-06, 4.055705142177693e-06, 3.870059768083771e-06, 1.8522695565345353e-06, 3.415001766067759e-06, 3.39789451726803e-06, 6.6742768883277406e-06, 5.9546279767282785e-06, 2.5163262541875798e-06, 3.347599469892966e-06, 1.6546790197924838e-06, 9.736112723300762e-06, 1.5632439203809224e-06, 1.7773395953488025e-06, 2.662963484226438e-06, 2.4735278546523498e-06, 4.09717994176534e-06, 6.108350622442167e-06, 1.5988214382860232e-06, 9.754220866026121e-06, 1.2796064671463145e-05, 2.4697711340327808e-06, 2.994773787176703e-06, 1.7384263371255223e-06, 2.43160121360976e-06, 1.5272955470197179e-06, 1.9127445238845444e-06, 4.57732848294763e-06, 4.229381009984566e-06, 8.207388898491101e-06, 2.3930004905840115e-06, 2.148485329113782e-06, 2.7680757956287638e-06, 5.64954481647878e-06, 2.7545106512406597e-06, 1.593134494224034e-06, 3.0417881122705642e-06, 3.0205695724603485e-06, 1.3369141106877514e-06, 2.407059210554043e-06, 2.2570964423199535e-06, 2.4149543577540072e-06, 1.7443786656437567e-06, 3.5457063370254053e-06, 2.196836568453815e-06, 2.1894292642283055e-06, 1.668207836292441e-06, 4.734172225818824e-06, 7.554405495131169e-07, 1.668207836292441e-06, 1.3786261022583087e-06, 2.3930004905840115e-06, 2.5830932505610206e-06, 1.6363723093740647e-06, 3.1764067084441464e-06, 6.603258251986599e-06, 4.091609114068383e-06, 8.842454226594659e-07, 2.1693976023524995e-06, 3.2965908981518973e-06, 4.559040391741763e-06, 1.0816810166917076e-06, 1.746833886416883e-06, 2.3405728741128703e-06, 1.287393707686287e-06, 1.5395666268134882e-06, 1.9392970638869937e-06, 1.7456288703642025e-06, 2.51337249989927e-06, 1.1159760685748193e-05, 5.086599397940603e-06, 6.037890011672988e-06, 1.0794600369920084e-05, 6.848993820434922e-06, 2.7118671766867165e-06, 1.2910780106645912e-05, 2.4636020388700215e-05, 7.188778440391034e-06, 2.6258335370958146e-06, 3.036665175086965e-06, 2.567414812266413e-06, 1.2802545818502079e-05, 3.044841717103236e-06, 2.17379243321668e-06, 2.17379243321668e-06, 3.929646436151164e-06, 2.229026842953514e-06, 4.806507944104108e-06, 5.258023783596699e-06, 3.3207185019313584e-06, 2.17379243321668e-06, 2.0014012004022417e-06, 2.542135099642292e-06, 3.960801116104007e-06, 2.7974648692648056e-06, 1.1805313537091206e-06, 7.6655377046201e-06, 8.218046917554822e-06, 4.553544587639442e-06, 1.1402489562689777e-06, 1.7972411466156033e-06, 2.238870454733273e-06, 5.341827777781594e-06, 3.5741886330784905e-06, 2.994773787176703e-06, 1.5520395220301484e-06, 4.348918569388488e-06, 6.091090828507381e-06, 4.6153893046208295e-06, 1.541075906748549e-05, 4.6153893046208295e-06, 1.0091407912046246e-06, 0.00017606159385050721, 5.289248011235544e-06, 3.206489565915115e-06, 1.1544949138868544e-05, 3.4244530097747337e-06, 3.826708221465637e-06, 8.313058496166932e-06, 5.331041326303336e-06, 2.6318306431620736e-06, 8.80667637517358e-06, 4.666456849701477e-06, 1.0113027694390156e-06, 6.915930249181221e-06, 3.2753373223414045e-06, 4.818028832548693e-06, 1.397663350281731e-06, 3.746738246344581e-06, 2.3101413125865327e-06, 3.0639518637301528e-06, 1.9106922980666496e-06, 4.5245465787444935e-06, 5.3670354085028285e-06, 3.645525225159423e-06, 1.4252624662034121e-06, 3.0025069012429168e-06, 2.331672598977283e-06, 3.315943943001797e-06, 9.526023710102363e-06, 3.3991429401429352e-06, 4.990851967519228e-06, 2.493337603443558e-06, 1.7463837055623195e-06, 1.1607479604448559e-06, 8.982361370620212e-06, 3.4373417223805927e-06, 1.557032711868583e-06, 1.8586745372319847e-06, 4.141392942082015e-07, 1.9798647425101367e-06, 1.79913601520104e-06, 1.6335166291992725e-06, 1.772621259716568e-06, 2.0590129835674717e-06, 3.3381178701666953e-06, 3.592462735794293e-06, 3.3381178701666953e-06, 3.592462735794293e-06, 8.957733161794515e-07, 1.9883480989237617e-06, 4.523667784079295e-06, 5.5546308072131344e-06, 4.356691334812159e-06, 3.2900648721456165e-06, 3.2997252818801253e-06, 2.0519969817919148e-06, 2.4128178110293877e-06, 3.243131060242439e-06, 2.17379243321668e-06, 1.3874376972231698e-06, 2.4876893864481234e-06, 2.6927632264743877e-06, 7.981004487248734e-07, 5.370287298224138e-06, 1.5493099624406927e-06, 2.5710791156378683e-06, 1.536018383125624e-06, 4.471355593800065e-06, 1.1276701682181046e-06, 2.2262938126788386e-06, 2.8256311037511225e-06, 2.2262938126788386e-06, 1.6627104976079033e-06, 2.3471507104485326e-06, 1.5060962366533033e-05, 3.73894176346251e-06, 3.0346452594710114e-06, 2.8004734837740837e-06, 1.7215556460195722e-06, 1.185319852652491e-06, 1.8224081731493182e-06, 2.1331848385632147e-06, 1.1307463058471174e-06, 2.652119390241587e-06, 1.7414548223486263e-06, 8.742652933346053e-07, 8.277590037654671e-07, 1.947077979655999e-06, 2.687481805376243e-06, 1.7850483163985783e-06, 3.347599469892966e-06, 2.2585864682366485e-05, 3.2923570228387403e-06, 1.8040192300483858e-06, 3.592462735794293e-06, 2.7171795070109465e-06, 2.1316334969945323e-06, 3.5319010513896683e-06, 3.0365119362625164e-06, 4.349718908113036e-07, 2.5513669461698953e-06, 1.8982994616916848e-05, 3.188063616823079e-06, 1.1763122347127615e-05, 5.690478953290663e-06, 2.00908738879866e-06, 4.92251923511261e-06, 2.17379243321668e-06, 5.2196334130873155e-06, 2.725172237669254e-06, 3.006653805472326e-06, 1.6877397991933882e-06, 3.201548998957113e-06, 1.958173054689388e-06, 8.452906384681778e-07, 7.118691538613966e-06, 1.124231687282804e-06, 3.406616985244764e-06, 1.04463655200494e-06, 1.3414541946924165e-06, 2.272478481778325e-06, 1.1430464118920091e-06, 1.862615688300101e-06, 1.5212911060849287e-06, 4.4036639253602696e-06, 3.806913249788508e-06, 1.0590484145596294e-06, 6.483601017499282e-06, 2.651847463578449e-06, 2.2536578118803663e-06, 5.375181641909504e-06, 1.1478621278597543e-06, 2.7092612002546383e-06, 2.407947886127743e-06, 5.266735529424107e-06, 9.072241670348486e-07, 3.346411669616368e-06, 3.346411669616368e-06, 6.336532160831447e-07, 4.9313743917748216e-06, 3.346411669616368e-06, 1.2835128636932948e-06, 4.34773845955688e-06, 1.419936337462423e-06, 2.7624545079622896e-06, 3.9084912767813735e-06, 3.537514860796029e-06, 5.1180389997466245e-06, 2.0738032314291064e-06, 2.373866798874042e-06, 2.62885311006309e-06, 2.141721430645982e-06, 2.674646143332951e-06, 2.373866798874042e-06, 3.250764640378592e-06, 1.6434059424439683e-06, 5.537956822735487e-07, 2.373866798874042e-06, 1.8704223142733762e-06, 3.980477262828681e-06, 5.528479700686049e-06, 1.8080019312963982e-06, 2.3712242886956193e-06, 5.063913562788651e-06, 3.0738998960827678e-06, 1.816239715358068e-06, 4.354802408745495e-06, 2.8101565108683094e-06, 2.6333114381417552e-06, 8.449185032608634e-07, 1.6772064091210337e-06, 1.7152252333567553e-06, 2.1979698469054876e-06, 3.313196302447129e-06, 2.275527085782783e-06, 1.786426963486366e-05, 4.405172771802629e-06, 3.873536790823329e-06, 2.7258364041152115e-06, 1.3631700685584437e-06, 4.86402506566441e-06, 0, 3.988569194417801e-06, 3.4371695353292303e-06, 1.946017528737314e-06, 2.4735278546523498e-06, 4.5753189173960344e-06, 2.0942926983212457e-06, 4.076525746505314e-06, 2.403968621114506e-06, 8.759494966013996e-07, 2.5969780275484583e-05, 1.7701720123742627e-06, 5.219762851662989e-06, 1.9344050137734774e-06, 1.3664946362657358e-06, 1.1654331450097883e-06, 1.7082405163662446e-06, 4.250168943454205e-06, 3.4860606134032856e-06, 2.969236287878239e-06, 1.7570889177278326e-06, 6.67297444981191e-06, 1.0391393162517192e-06, 1.4767485611591908e-06, 3.32191425393668e-06, 3.912880224094224e-07, 2.641738315483161e-06, 2.2312111097322357e-06, 2.7542656068046864e-06, 3.681872473016136e-06, 2.950710366401335e-06, 1.3640069827110993e-06, 4.845417079285714e-06, 4.7910943531339675e-06, 4.4760398717377355e-06, 3.7865177519803544e-06, 3.8077043263529855e-06, 2.729670450238303e-06, 2.6325175841371525e-06, 2.3733870513877765e-06, 2.034571909004734e-06, 3.5841599611041912e-06, 2.8936090534749744e-06, 2.44227993629147e-06, 2.4618846678672347e-06, 1.5188818712551157e-06, 2.218746886999654e-06, 1.6238605530243278e-06, 5.728476774026442e-06, 1.9537338218260907e-06, 2.218746886999654e-06, 2.3991490448444928e-06, 1.6849729174750771e-06, 3.609954598494596e-06, 2.459854203904666e-06, 3.872703886756115e-06, 1.1796788511857047e-06, 1.261579328829644e-06, 3.5635833635678043e-06, 2.088348555577453e-06, 2.5494700472801284e-06, 1.5413410858318116e-06, 3.84652110712481e-06, 1.4577320165583285e-06, 1.657616815690363e-06, 4.850982179057223e-06, 0, 2.5539387790174514e-06, 1.8079543995549522e-06, 2.3267055837347745e-06, 2.605309531712152e-06, 2.224179446359183e-06, 2.2308288752870325e-06, 1.4769735937149907e-06, 1.2704809477315065e-06, 2.0942926983212457e-06, 2.0279060181663807e-06, 4.338429905729069e-06, 2.0872887952418178e-06, 1.4163365592703802e-06, 9.36159728047729e-07, 1.9135652025336506e-06, 1.6363946680654638e-06, 2.18929027638125e-06, 2.490540643913242e-06, 2.18929027638125e-06, 2.899842887735169e-06, 2.18929027638125e-06, 6.852383213128364e-07, 2.3041722081029457e-06, 1.0160446057985719e-06, 1.9083711448266772e-06, 2.7723847658964845e-06, 1.6488505690346074e-06, 7.648639547929953e-07, 6.029567018090487e-06, 1.2900977307202848e-06, 2.3477563649878084e-06, 1.3386491842022493e-06, 2.437857238438794e-06, 3.4661357360464273e-06, 6.413121488753659e-06, 1.1916507394476385e-05, 3.7997145770993143e-06, 2.1940873646852876e-06, 9.443427073168767e-07, 5.985002142078625e-06, 2.51337249989927e-06, 1.4421522954733809e-06, 5.268670263146318e-06, 2.1960806596963207e-06, 1.7384263371255223e-06, 2.2303939272513947e-06, 9.50681437051291e-06, 1.2991194853446024e-05, 1.4955818899167015e-05, 3.1987155644595265e-06, 2.6924899142636264e-06, 2.3164114273286505e-06, 1.153635560428152e-06, 4.1981215401522045e-06, 3.323192430491778e-06, 3.2790151844750088e-06, 1.8335602568809146e-06, 1.5992170949176376e-06, 1.2027397869979033e-06, 2.516518162438391e-06, 1.9157569394245093e-06, 3.4137505900337885e-06, 5.635696298281434e-06, 3.6668387547745675e-06, 2.6986656760869485e-06, 5.739360468596763e-07, 4.0481563722012484e-06, 2.2739571886342022e-06, 1.2650605122687122e-06, 2.9027475331872067e-06, 1.0735727906685722e-06, 1.4488412715420339e-06, 1.2000142403670305e-06, 1.1297575636178102e-06, 1.4683536222378046e-06, 1.4488412715420339e-06, 1.8310271446225092e-06, 4.165748508249634e-06, 1.9089018271650893e-06, 2.19148299192974e-06, 7.799962225872473e-06, 1.4305093139328665e-06, 2.5883173960737353e-06, 4.703344742288797e-06, 5.528240107939892e-06, 2.6875804248251613e-06, 1.588623233116011e-06, 1.7948756544771819e-06, 1.2000142403670305e-06, 2.9417880051577717e-06, 1.4488412715420339e-06, 1.4488412715420339e-06, 4.824740112687733e-06, 1.7503668828148319e-06, 2.3156324537962967e-06, 1.6408360452507495e-06, 1.6042879833878671e-06, 1.6834993454618042e-06, 3.5686922715984066e-06, 1.1276701682181046e-06, 1.1433438199067894e-06, 1.4421522954733809e-06, 2.1685230892105756e-06, 1.506250261846421e-06, 1.6339998086511706e-06, 2.0942926983212457e-06, 1.7156893592168065e-06, 2.4996750055119626e-06, 2.4149543577540072e-06, 1.5051942217045223e-06, 3.347599469892966e-06, 3.3381178701666953e-06, 2.777791973204742e-06, 8.8304928239638e-06, 1.1296034166853933e-05, 1.7366609794420473e-06, 1.967262152880437e-06, 7.5618483523907035e-06, 7.36347237847189e-07, 9.016280736919063e-06, 7.1308764275565395e-06, 2.8694551077904873e-06, 2.8368353452289127e-06, 2.2739649215123866e-06, 3.0767830400327995e-06, 3.4653103735199147e-06, 2.8767569234856216e-06, 1.6293922849977722e-06, 2.5552970341379223e-06, 1.2301398722862553e-06, 3.969702090690598e-06, 3.592462735794293e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.064601997239956e-06, 1.50516539258982e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 9.580440018778554e-07, 9.678096202792122e-07, 1.2337066388358403e-06, 3.3057878300338494e-06, 2.5277680493927685e-06, 4.995673783061456e-06, 4.493332930952658e-06, 5.4990454674222155e-06, 1.7315636247497355e-05, 3.4431114318294805e-06, 3.3138214988008855e-06, 1.9986154255014246e-06, 2.992707815159298e-06, 4.239110833363694e-06, 6.747907505966646e-06, 1.850467404732725e-06, 2.4735278546523498e-06, 4.202087142909469e-06, 8.658351924682918e-06, 4.479739234257842e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 3.4711437032493457e-06, 2.3930004905840115e-06, 7.442660603325389e-06, 3.6119907455790856e-06, 3.6632835516258636e-06, 1.8776043742196325e-06, 3.469607083115679e-06, 2.1020582679665735e-06, 2.4227651523583438e-06, 2.6337168890548374e-06, 2.501212152674062e-06, 2.7667485607914795e-06, 9.473470653266027e-06, 4.146949467226952e-06, 1.7558824772271129e-06, 7.95979248891987e-06, 1.4781755300343189e-05, 5.834848395902638e-06, 1.5103072493631939e-06, 2.6544072821433725e-06, 6.529501112680199e-07, 6.63159396752151e-06, 3.7165271976200677e-06, 5.128191878230229e-06, 3.2767114024381794e-06, 8.882222617486673e-06, 1.131579920498113e-05, 4.752878593650587e-06, 7.297449762252134e-07, 3.982997958915863e-06, 2.93854897330431e-06, 6.2435503020952905e-06, 2.1871443690945535e-06, 1.2685806865277424e-06, 9.623993848082647e-07, 3.1645581712411835e-06, 8.957733161794515e-07, 3.296353937109032e-06, 3.223246653162998e-06, 1.474756204957915e-06, 5.111364629432999e-06, 2.0836176428119587e-06, 1.1360128878618686e-06, 9.814168317562704e-07, 2.1063096143721164e-06, 1.3004999029358367e-06, 1.8958575939263035e-06, 2.299686976843995e-06, 4.817563629430897e-06, 2.751625209550473e-06, 3.833413777521751e-06, 3.4244963534102212e-06, 2.090715679279853e-06, 6.220112524719e-06, 3.476315128317335e-06, 2.1125494113841094e-06, 2.4658322121203017e-06, 2.3775430343907356e-06, 4.012976991641324e-06, 1.86424615509876e-06, 4.3831952205406305e-06, 1.971529838964592e-06, 2.2565111817547776e-06, 8.068345168908953e-06, 3.048211390892295e-06, 5.329721769159807e-06, 1.3250587649857364e-06, 1.390088016632348e-06, 4.580143818613655e-06, 1.0076994898223477e-06, 5.7614619366097085e-06, 8.956625948830082e-07, 1.0126322390978274e-05, 3.137662333820318e-06, 2.130686835000308e-06, 3.2936631536998e-06, 2.8599249155747324e-06, 4.356601977950818e-06, 1.2883406128916802e-06, 6.711901786080303e-07, 1.5055871882083838e-06, 1.062404075313477e-06, 3.592462735794293e-06, 1.5562522099845549e-06, 2.141721430645982e-06, 1.2097083426092563e-05, 2.1178918539518284e-06, 5.249736973946295e-06, 3.775462933026986e-06, 6.488608994431603e-06, 4.0162998170834764e-06, 4.99195359423696e-06, 3.316491772078311e-06, 6.277644329240057e-06, 5.505017666857731e-06, 5.2721805434349586e-06, 4.181934851019945e-06, 2.3450980033581487e-06, 2.706393663753698e-06, 2.5385375191540056e-06, 3.547193843204833e-06, 2.601878968715823e-06, 5.535563165074723e-06, 1.6023075367001224e-06, 3.291607675253376e-06, 1.446558310802488e-05, 4.136460031207519e-06, 5.084723446342139e-06, 3.6021410536326798e-06, 8.992984412383848e-06, 5.589022071734351e-06, 2.681430153717303e-06, 9.212864626666829e-06, 3.833413777521751e-06, 1.021100126814501e-06, 1.752656397034738e-06, 2.2986087795085028e-06, 5.319571807732331e-07, 1.0076994898223477e-06, 6.726609671831609e-06, 8.03892526618229e-07, 1.050689874198246e-06, 6.253779409703976e-06, 5.709815859107118e-06, 5.32887698085355e-06, 4.414574252535387e-06, 9.498268528898032e-06, 2.241408193611128e-06, 2.185715584714717e-06, 6.340071295439115e-07, 2.5120156905774175e-06, 1.506250261846421e-06, 4.606593412600118e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.4421522954733809e-06, 1.506250261846421e-06, 8.735125707620576e-07, 2.5120156905774175e-06, 2.6176040451123986e-06, 2.54311374387837e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.1204396988712402e-06, 1.118001067013821e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.554324851976409e-06, 1.506250261846421e-06, 6.978588492844929e-07, 2.5120156905774175e-06, 1.50756864122747e-06, 2.1292217017757087e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 2.32846503721744e-06, 2.5120156905774175e-06, 4.6916207893045645e-07, 1.506250261846421e-06, 3.427752139982371e-06, 1.855587054704814e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.93854897330431e-06, 7.677029372490281e-06, 1.6548091761726205e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 4.341211484310321e-06, 1.5222459436311788e-06, 1.9638211512535317e-06, 1.1340960292297039e-06, 1.7075915859652574e-06, 3.5308481082628633e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 2.479938732535084e-05, 1.4783555747425507e-06, 1.3616375577464776e-06, 2.713650881948908e-06, 1.644840607982074e-06, 1.506250261846421e-06, 5.948171835706272e-06, 2.5598091595109298e-06, 1.3414541946924165e-06, 1.50756864122747e-06, 3.7450800817703735e-06, 3.962178121114571e-06, 6.017768914310406e-06, 1.2715561610370712e-06, 3.1447828481826677e-06, 8.452906384681778e-07, 3.3263389691044235e-06, 1.2269697239143592e-05, 1.5901774097591193e-06, 8.485465120921393e-06, 1.2715561610370712e-06, 5.885031705691688e-06, 8.261558665876716e-07, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 8.798652492630945e-07, 4.684437218064053e-06, 2.8317167627248454e-06, 1.7839761875546574e-06, 3.535340411226079e-06, 6.62285415273463e-06, 2.5120156905774175e-06, 7.677029372490281e-06, 2.3580315594683893e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.6957037635273038e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 1.8258773002746345e-06, 5.755625262452559e-06, 1.8258773002746345e-06, 2.1685230892105756e-06, 2.242953464711121e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5495597013762253e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 4.286778005093873e-06, 2.743973065215624e-06, 3.1671801632005923e-06, 4.457706080828437e-06, 2.8846406185227507e-06, 1.4794776275836578e-05, 8.840608248634763e-06, 2.5120156905774175e-06, 4.81685474391672e-06, 4.519889767412154e-06, 3.2306495329823085e-06, 1.3691314650834576e-06, 1.3137584366303772e-06, 5.2951705296911084e-06, 7.124046153716738e-07, 1.2715561610370712e-06, 8.555171582268892e-07, 8.862610488777212e-06, 6.260802706329998e-06, 1.8417570368294298e-06, 4.978020464017566e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.551740129176075e-06, 1.982965751492052e-06, 2.1292217017757087e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 4.6916207893045645e-07, 1.6224403765833176e-06, 1.982965751492052e-06, 2.141721430645982e-06, 2.580168692689252e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.7828561994526255e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 3.7251482866834075e-06, 1.1507473993738775e-06, 3.4593462703596692e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 1.8588799648678779e-06, 3.5606080280879963e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.984747512037354e-06, 2.6815264110948968e-06, 3.873564494369507e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.544957931974505e-06, 5.149036103675271e-07, 2.472225023816832e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 4.6916207893045645e-07, 1.7783898368514997e-06, 1.0782842332298494e-06, 1.2729798705650078e-06, 1.7805596275371022e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.5940495372909991e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 2.3723777772354266e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 4.6916207893045645e-07, 2.8227476657566644e-06, 3.3484054595881687e-06, 1.7125428189007855e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 1.6548091761726205e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.7402616719266454e-06, 2.1685230892105756e-06, 5.4743712925625765e-06, 2.93854897330431e-06, 1.8258773002746345e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.3817384858653223e-06, 1.1885096845669891e-06, 4.4162669827778205e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 2.7008004719429105e-06, 1.506250261846421e-06, 2.5120156905774175e-06, 1.506250261846421e-06, 9.233084000922082e-07, 1.0052639062281257e-06, 1.8195351069032484e-06, 1.506250261846421e-06, 1.506250261846421e-06, 1.8425476510867212e-06, 7.231141091667221e-07, 1.506250261846421e-06, 6.340071295439115e-07, 9.917762583916338e-07, 1.506250261846421e-06, 1.506250261846421e-06, 1.506250261846421e-06, 2.3900444028282823e-05, 1.4733202959836066e-06, 4.66770126137036e-05, 1.8484883115950635e-06, 2.687481805376243e-06, 4.6249028954124275e-06, 7.02301089756749e-06, 1.5895733355070989e-06, 5.686701149344895e-06, 1.2715561610370712e-06, 1.0854121124226695e-05, 6.724790306539747e-07, 7.919764918350737e-07, 9.85364709589816e-07, 2.601878968715823e-06, 2.145656424066041e-06, 6.516700535948718e-06, 3.238139377218194e-06, 4.617732705070182e-06, 4.788974215151716e-06, 1.4803082321658089e-06, 2.416596942776661e-06, 1.506250261846421e-06, 5.793434171763893e-06, 1.2095607113369237e-05, 7.65013405107731e-06, 6.502258992054016e-06, 1.506250261846421e-06, 3.1964685408225317e-06, 1.2839283445594568e-06, 1.752332495838245e-06, 1.9986154255014246e-06, 1.506250261846421e-06, 4.425644549776805e-06, 1.554324851976409e-06, 1.865531630822665e-06, 1.362483462427037e-06, 1.362483462427037e-06, 1.4564421834786356e-06, 3.592462735794293e-06, 2.4110439590589672e-06, 1.5648350953772786e-06, 8.211755504926682e-06, 5.4504140146291004e-06, 1.2418347028561752e-05, 2.9854971446824467e-06, 2.969236287878239e-06, 2.8812631499826653e-06, 2.1756303975763306e-06, 1.860315492538921e-05, 2.2996867313617847e-06, 2.0281131293653217e-06, 2.221311135564517e-06, 8.976619172825884e-06, 5.985234027633983e-06, 2.6399270864364576e-06, 2.5865991782604357e-06, 5.465884384062901e-06, 4.392694473269727e-06, 2.992707815159298e-06, 2.992707815159298e-06, 3.920535736696471e-06, 3.8140338189321025e-06, 5.701005902746164e-06, 4.53471485438139e-06, 2.635168887857025e-06, 1.2479085404486097e-05, 3.029671507489449e-06, 7.532661229381158e-06, 1.1547088713064594e-05, 5.110100200683895e-06, 3.609821765992029e-06, 1.7701720123742627e-06, 4.005617149124557e-06, 3.4137505900337885e-06, 3.775286147787732e-06, 2.141721430645982e-06, 2.141721430645982e-06, 2.1650187549418342e-06, 1.7233590493546028e-06, 3.4978993429558317e-06, 1.506250261846421e-06, 3.732664427098406e-06, 6.297228327190395e-06, 2.969236287878239e-06, 2.8513396403401315e-06, 6.9709221018014965e-06, 3.5134439367560403e-06, 1.7433673320119248e-05, 2.8599313265741294e-06, 7.999208409789991e-06, 6.191374995374816e-06, 3.833413777521751e-06, 2.5770889543528297e-06, 4.82029819870196e-06, 2.9611764687144713e-06, 4.6916207893045645e-07, 1.5940495372909991e-06, 1.4132345994224553e-06, 3.6688864534993387e-06, 2.882331390163109e-06, 3.581417425845748e-06, 2.67423129219582e-06, 1.1871019108194093e-06, 3.267522597165971e-06, 1.9406159202468595e-06, 2.647931886202942e-06, 3.1950540022436396e-06, 2.1606763572040634e-06, 2.052758764615457e-06, 1.421404576937233e-06, 3.159068512800925e-06, 5.462170500699809e-06, 1.4030309202632454e-06, 2.331442843217194e-06, 1.808915754086957e-06, 5.443557965061724e-06, 4.3284474337207974e-06, 4.6916207893045645e-07, 1.9943762399746887e-06, 4.955798263523921e-06, 1.6656262816184481e-06, 1.8260706562417388e-06, 1.017202177372458e-05, 2.3561013365940013e-06, 1.6458259069954282e-06, 4.596847279577607e-06, 2.7371946067659058e-06, 4.6916207893045645e-07, 1.5554248971734024e-06, 1.5940495372909991e-06, 1.0782842332298494e-06, 5.886044994901694e-06, 1.982088165884304e-06, 2.84091172773801e-06, 4.6916207893045645e-07, 1.5940495372909991e-06, 1.8012564032172732e-06, 1.6063441677059346e-06, 4.322878041397706e-06, 3.638149781921122e-06, 1.5282808609987584e-06, 3.481208269752858e-06, 1.88074522110699e-06, 8.840608248634763e-06, 5.057232502091891e-07, 2.298100201401491e-06, 2.696670736239289e-06, 1.909348275737044e-06, 4.234228437325903e-06, 4.728797955159068e-06, 1.8027747659076015e-06, 2.4735278546523498e-06, 1.237820805173495e-06, 6.85828783428718e-07, 3.7149061963035654e-06, 6.367384826831425e-06, 5.2403063406042735e-06, 1.0761491439485788e-05, 1.669143497954208e-06, 1.7986908973381516e-06, 1.669143497954208e-06, 2.416812977086863e-06, 2.8599313265741294e-06, 1.2748988917945688e-05, 4.914641834895396e-06, 4.233590667628745e-06, 6.675773917614871e-07, 2.774013045040797e-06, 9.078608114682393e-07, 4.6778571136148175e-06, 1.0582317102721573e-05, 2.8751148979189473e-06, 2.8751148979189473e-06, 3.7931896128381776e-06, 5.1887795655248255e-06, 6.855710405488556e-07, 1.3468411481681142e-06, 1.4328760197933398e-06, 2.0441191712898265e-06, 2.0014662705359974e-06, 7.404260604523373e-06, 2.0102353635511826e-05, 1.563971785685711e-06, 1.816239715358068e-06, 2.768769460445123e-06, 2.77505619905496e-06, 1.8455012943822276e-06, 3.1551066197390808e-06, 3.1961841793043295e-06, 2.348708244522015e-06, 3.177026537666802e-06, 4.101566732792701e-06, 2.957450266098311e-06, 1.0949315390218585e-05, 1.2117233810890634e-05, 5.178306446160545e-06, 2.932090725972334e-06, 3.8237982855008135e-06, 6.4861648575949565e-06, 3.0170122792574136e-06, 3.354777847533825e-06, 3.8237982855008135e-06, 2.215700753176978e-06, 5.276609514768066e-06, 9.207967271174699e-07, 1.2685806865277424e-06, 9.474873944944642e-07, 1.8637269493306277e-06, 2.8202788950219463e-06, 2.713650881948908e-06, 3.2560570658407965e-05, 6.083071168864086e-06, 4.278448641353603e-06, 6.711901786080303e-07, 2.992707815159298e-06, 3.7614909390937955e-06, 7.372670181209823e-06, 2.6623313537336585e-06, 1.6660957366635068e-06, 2.6623313537336585e-06, 1.7292856308496413e-06, 2.584175026643329e-06, 3.0751891402356456e-06, 7.912946421863036e-07, 5.3901004728303255e-06, 2.911603611205507e-06, 4.5352220417842255e-06, 1.6597904696983907e-05, 2.7006886964379634e-06, 4.073677549688717e-06, 2.2802483947325228e-06, 2.0799068931237427e-06, 1.0930986235424947e-06, 1.7010549684000454e-06, 3.1086080191450143e-06, 3.1742483024231664e-06, 6.558030735212233e-06, 2.5125007778382533e-06, 2.8031230382551477e-06, 7.438746579843129e-06, 5.420333000731101e-06, 4.661753289989245e-06, 4.677051886618266e-06, 5.033036836221616e-06, 2.307253551957865e-05, 1.0523874511445968e-05, 1.670790130857639e-06, 3.419603400468875e-06, 3.547193843204833e-06, 2.0755700739482363e-06, 1.8185490290905164e-06, 3.1395793561658787e-06, 2.992707815159298e-06, 2.5325099555411597e-06, 2.036830616407238e-06, 5.2830379186232585e-06, 1.4469960972062186e-05, 5.475583681915893e-06, 6.607014570268824e-06, 3.390780912689704e-06, 3.6670744718138024e-06, 4.458916651214881e-06, 3.936857966500977e-06, 1.35225258004159e-05, 3.5860903966889975e-06, 2.1831559611374425e-06, 1.147928762562552e-05, 1.5471535591551082e-06, 1.3641688603542653e-06, 3.070688749267359e-06, 3.1063424692595707e-06, 2.3303795488993937e-06, 3.3771925073430624e-06, 1.293968668988482e-06, 1.329610183019009e-06, 5.6208648840821325e-06, 2.7754963481409334e-06, 2.784852035925739e-06, 3.4508018695907546e-06, 2.366147761181227e-06, 2.2025356209225202e-06, 3.8086985917456487e-06, 1.934044398099013e-06, 3.599292054038941e-06, 5.648361380515888e-07, 7.052598933144579e-06, 2.445715591436451e-06, 5.528414195233933e-07, 5.384054084184917e-06, 3.056598830426481e-05, 5.564148346242445e-06, 5.045829094578797e-06, 8.74127055481549e-06, 4.245401605888349e-06, 7.033993903168625e-06, 4.513296944285341e-06, 3.584728468718795e-06, 1.8778041242913535e-06, 1.0076994898223477e-06, 2.037787939017923e-06, 1.2301398722862553e-06, 2.6238949979137537e-06, 1.6232810553298986e-06, 5.0866113304498236e-06, 2.0014662705359974e-06, 1.062404075313477e-06, 1.3066567171387638e-06, 9.172305093306261e-07, 4.322522476117209e-06, 4.356601977950818e-06, 2.4618846678672347e-06, 1.563971785685711e-06, 2.435700392045468e-06, 2.9940724002599803e-06, 2.180489326246408e-06, 1.788499508171965e-06, 7.2589874179347405e-06, 5.9915666776304395e-06, 1.8012564032172732e-06, 1.7441586380323302e-06, 1.8012564032172732e-06, 1.4959012185605627e-06, 2.7673623014095734e-06, 3.809490717050168e-06, 3.592462735794293e-06, 1.2268959129169455e-05, 3.343760036013058e-06, 1.8508124933584568e-06, 3.2893262455064387e-06, 1.1357030813497736e-06, 5.029583770969965e-06, 2.4107075643104414e-06, 5.293020747399001e-06, 1.5056883963327125e-06, 6.275346182706477e-06, 1.3197121835545451e-06, 7.741278503106134e-06, 9.39312058204382e-07, 2.665154886761694e-06, 1.276689151363417e-05, 2.6041564011606586e-06, 8.733491427960858e-06, 9.70891563130638e-06, 4.096972702134983e-06, 2.7007251856483783e-06, 1.816239715358068e-06, 3.592462735794293e-06, 4.646655378289563e-06, 2.4735278546523498e-06, 4.262591092299505e-06, 7.781998951021606e-06, 2.5176323328682577e-06, 2.51337249989927e-06, 1.8987127677974501e-06, 2.51337249989927e-06, 2.961552734236429e-06, 4.73848938351326e-06, 6.1858010333601245e-06, 8.585507546819893e-06, 4.781124771732147e-06, 1.1510570619779515e-06, 9.322909261109603e-07, 1.790039955555006e-05, 1.0816810166917076e-06, 1.943614337730763e-05, 4.614516157710922e-06, 3.134722237654856e-06, 1.8307354377687106e-06, 2.831109774755142e-06, 1.5415356218066338e-06, 7.261778805338084e-06, 6.184133443212086e-06, 5.975192421414671e-06, 6.741124472544271e-06, 5.797439669595501e-06, 2.8325867552348874e-06, 8.452906384681778e-07, 3.604929418587107e-06, 1.0357525886009704e-06, 5.8374318223433844e-06, 3.8391224461874235e-06, 6.365752123722957e-06, 3.639693498239279e-06, 1.4826457006086956e-06, 6.227098391644013e-06, 9.941480646386912e-06, 2.3930004905840115e-06, 3.1750744487603014e-06, 2.788808386262567e-06, 8.452906384681778e-07, 4.0918862285350955e-06, 1.506250261846421e-06, 1.5098182817533078e-06, 2.855986657543398e-06, 3.7902355170613664e-06, 1.4175377615457343e-06, 1.677564517260555e-06, 1.3057209404067357e-06, 1.8260706562417388e-06, 3.0017256107703946e-06, 1.616233560570536e-06, 5.605227588573112e-06, 3.644189146197529e-06, 2.670049347161753e-06, 1.0427043738705337e-05, 4.665025824793196e-06, 9.177139095164462e-06, 3.104076613975255e-06, 2.438041043043222e-06, 5.239558194793486e-06, 2.1577186354628015e-06, 1.6216157122187254e-06, 1.5529026389526823e-06, 5.923542531524248e-06, 9.225220620479023e-06, 3.7986922544073926e-06, 3.725823396586812e-06, 1.0690919292384617e-06, 1.6403922186334296e-06, 3.1447828481826677e-06, 3.713783559809774e-06, 6.084576717944297e-06, 1.2895385843325234e-05, 6.2977031128108944e-06, 2.0322165665410325e-06, 2.649212829285555e-06, 1.0028971864393445e-06, 2.045581269265493e-06, 6.243855919448437e-07, 1.7870942712353562e-06, 4.380602549910539e-06, 3.238378326378213e-06, 4.3535780800028895e-06, 1.461233414644025e-06, 4.258070779179634e-06, 3.3057088868147227e-06, 7.2949644825237266e-06, 4.7720488163108605e-06, 2.3985416539792135e-05, 8.042413064854399e-06, 1.7140595867812297e-06, 3.4365787660653538e-06, 2.8751148979189473e-06, 1.9743182640120114e-06, 1.7186941173403902e-06, 1.4721560525485437e-06, 1.0292150833557469e-06, 1.2304444469455551e-06, 1.4357044576796512e-06, 3.1722976303184808e-06, 4.00729802633668e-06, 4.267699605884182e-06, 1.756008945711344e-06, 3.0246787860535256e-06, 2.3839458231200653e-06, 4.57732848294763e-06, 6.319567536149386e-06, 4.633113909109453e-06, 3.0688689195192083e-06, 8.929266199621345e-06, 2.8982077940218206e-06, 3.1814555229353304e-06, 2.4531297127737653e-05, 1.2957496258162228e-06, 3.346411669616368e-06, 2.7146296504480254e-06, 1.913065751533111e-06, 1.6656477192148137e-06, 5.314399330773606e-06, 1.4836747640016584e-05, 1.5262406714418108e-06, 3.346411669616368e-06, 3.346411669616368e-06, 2.1010729308350876e-06, 1.588961870142485e-06, 1.4529881825277723e-06, 2.48025061896019e-06, 2.997029064347864e-06, 2.2083171952587003e-06, 1.597443525154092e-06, 7.495086136242349e-06, 2.0415945827796092e-06, 4.131809727072874e-06, 1.3293484349218992e-05, 2.7337373289142345e-06, 5.093947243942691e-06, 1.4790136470960347e-06, 1.5397496154203278e-06, 2.0881628373743157e-06, 1.182747981420331e-06, 1.5667583313946124e-06, 7.489120675554291e-07, 6.077229053367373e-07, 2.495931059629363e-06, 3.0673511059349076e-06, 2.714445690530456e-06, 1.0243167078833597e-06, 2.27191590081862e-06, 3.05000348266114e-06, 4.304835280207095e-06, 1.4932765372789054e-06, 2.601878968715823e-06, 1.1487137093032727e-06, 3.841848029885558e-06, 2.6623313537336585e-06, 2.6623313537336585e-06, 2.0513682389127853e-06, 1.0706257415432478e-05, 1.2135344945750556e-06, 1.2399289418584916e-05, 5.9341783772154425e-06, 3.2281669628582456e-06, 1.5936905923148998e-06, 4.000342658422623e-07, 3.2592757216595966e-06, 2.9364525590377033e-06, 3.7902512480245967e-06, 1.619587102360744e-06, 1.2715561610370712e-06, 6.179401925791798e-07, 5.8125158455739085e-06, 4.144632484463815e-06, 3.31389603706922e-05, 1.850467404732725e-06, 2.506988162525998e-06, 3.3208983850641054e-06, 1.8540538416996203e-06, 4.883908777780673e-06, 7.0267030136062865e-06, 4.559040391741763e-06, 5.1294714216399226e-06, 5.793027247341689e-06, 4.5420445428660885e-06, 2.1009274095569257e-06, 7.677029372490281e-06, 1.6670945815201708e-06, 1.8258773002746345e-06, 6.447296903301068e-06, 3.963494055310106e-06, 2.7600815619687956e-06, 2.2532726826904242e-05, 3.258891149813384e-06, 8.889954956621505e-06, 9.40886845023016e-06, 4.138891037973263e-06, 1.5112551468061454e-06, 3.219472729264794e-06, 8.069918460172155e-07, 3.1560785266145702e-06, 1.4067393144617713e-06, 7.662615886792939e-05, 2.182672994083695e-06, 3.349701508361456e-06, 2.7886600535682536e-06, 5.493833834828753e-06, 3.146709923231088e-06, 3.3917412557677374e-06, 6.575358167722291e-06, 1.5992169973691168e-06, 1.9305007550095468e-06, 2.652119390241587e-06, 1.0113027694390156e-06, 4.325348073664557e-06, 1.7850483163985783e-06, 1.9087320672768942e-06, 1.390088016632348e-06, 3.815066272399505e-06, 1.8969209972628155e-06, 4.115443392477838e-06, 1.6585199808267932e-06, 2.3277968064800267e-06, 6.863437505252642e-06, 2.057566574610584e-06, 2.155410868366408e-06, 1.6522998066287284e-06, 2.393467632058091e-06, 2.960992056457648e-06, 2.145656424066041e-06, 1.6808666747968931e-06, 2.078195684080466e-06, 1.3137849219143685e-06, 2.177035503110417e-06, 2.7464098017072377e-06, 3.346411669616368e-06, 2.5946051856817342e-06, 3.346411669616368e-06, 1.7575876089012774e-06, 2.665962404280764e-06, 3.346411669616368e-06, 1.7441586380323302e-06, 1.7441586380323302e-06, 1.8915722084906928e-06, 2.3810361707804124e-06, 2.102076411268966e-06, 2.3797741721485433e-06, 1.780870387850146e-06, 4.1883495607869745e-06, 4.443778372650204e-07, 1.8858499396553344e-06, 4.9758173805245554e-06, 7.975171319551145e-06, 1.749135610180112e-06, 3.246472237835106e-05, 5.718819164028026e-07, 1.118001067013821e-06, 8.309998848265336e-06, 2.8865571264771114e-06, 2.254926241203423e-05, 3.01478329469763e-06, 3.6434317817468337e-06, 1.7820982637902424e-06, 1.4544286282199722e-06, 3.4322249774231847e-06, 9.79856337910162e-07, 2.3681344244605155e-06, 1.3684366393198965e-06, 1.2444973668490133e-05, 1.2013904207436268e-06, 1.848195908554683e-06, 2.3024717637021418e-06, 1.9261523361199417e-06, 8.900617154706959e-07, 1.3366733599750773e-06, 5.526490434687068e-06, 5.278350814723236e-06, 1.8655568361624373e-06, 1.6217222964038626e-06, 5.008483451286649e-06, 3.916290368986344e-06, 1.2149841336057293e-06, 5.422675838888485e-07, 1.1157802399381009e-05, 5.5714844750674395e-06, 3.7217100986408498e-06, 1.2295954964275378e-06, 4.109007940226845e-06, 1.2974807586598697e-06, 3.391987151458013e-06, 2.600084118072717e-06, 3.395733859052285e-06, 2.0014662705359974e-06, 2.8451349998487915e-06, 2.1168771973562834e-06, 2.5216079331456113e-06, 2.625358596694389e-06, 7.5575534996019154e-06, 2.803727318037312e-06, 2.141721430645982e-06, 1.4027893869256581e-06, 7.96852263058168e-07, 1.9170184138303123e-06, 7.96852263058168e-07, 2.8943919079046487e-06, 5.8933505825003e-05, 8.645071362037552e-06, 1.612114059392641e-06, 3.547193843204833e-06, 3.778997207260135e-06, 2.6166161276638072e-06, 3.1395793561658787e-06, 4.775342984690781e-06, 3.3057088868147227e-06, 4.3535780800028895e-06, 3.6997913820495192e-06, 4.00379386024726e-06, 1.7441586380323302e-06, 1.612114059392641e-06, 2.141721430645982e-06, 3.6032877466850312e-06, 1.746833886416883e-06, 3.7321234325631262e-06, 3.447831228130388e-06, 1.8342342902001696e-06, 1.3591061645155371e-05, 4.4387939345814344e-07, 1.707920916760784e-06, 2.803727318037312e-06, 1.062404075313477e-06, 2.221311338806742e-06, 7.499878943956288e-06, 2.048788020131023e-06, 2.008590542019641e-06, 2.0922532213868157e-06, 1.6063441677059346e-06, 1.7441586380323302e-06, 2.100220035252406e-06, 2.47552200185435e-06, 2.1700674187958928e-05, 2.4115928060349806e-06, 2.297210799097531e-06, 2.0922532213868157e-06, 1.1401220647314831e-06, 2.5431859551363334e-06, 1.612114059392641e-06, 2.0922532213868157e-06, 1.8012564032172732e-06, 1.612114059392641e-06, 3.337061262985569e-06, 1.9357406085812983e-06, 2.425277324676134e-06, 5.013527752661965e-06, 6.280654815439391e-06, 3.395294353675316e-06, 3.675531878138077e-06, 5.352099290222919e-06, 1.8012564032172732e-06, 2.0014662705359974e-06, 1.563971785685711e-06, 2.3600948317448597e-05, 3.287059830703697e-06, 3.942106902676839e-06, 2.6623313537336585e-06, 6.085701298502714e-06, 5.34050601223071e-06, 2.5441007956338216e-06, 1.0459062350988177e-05, 3.663952236224717e-06, 1.8449518594518907e-06, 5.332086587983859e-06, 3.316491772078311e-06, 4.320450377580704e-06, 2.7458112177547288e-06, 1.9289507658400183e-06, 8.24162663674201e-06, 1.4110970217005102e-05, 2.3175347178937755e-06, 2.4226586031302385e-06, 3.547193843204833e-06, 2.2468295622625833e-06, 2.2468295622625833e-06, 2.2468295622625833e-06, 2.454225099928455e-06, 2.5521090079272362e-06, 1.7701720123742627e-06, 2.8751148979189473e-06, 2.6623313537336585e-06, 1.674469256727895e-06, 3.980871407205178e-06, 2.1843623029939e-06, 3.2249809460778e-06, 1.689367370953051e-06, 2.635168887857025e-06, 1.7701720123742627e-06, 2.5105837318084753e-06, 1.4567017292413366e-06, 1.3558129905870715e-06, 1.9986154255014246e-06, 3.1039214998652804e-06, 2.221311135564517e-06, 2.7720994513810843e-06, 2.8751148979189473e-06, 1.1290044279426455e-05, 8.39798991076952e-06, 5.395803197591969e-06, 6.528522667655687e-06, 2.3297713781264237e-06, 3.341415911726057e-06, 1.796322428745053e-06, 1.0161212734453942e-06, 2.3009076629758496e-06, 1.970292193992187e-06, 4.356601977950818e-06, 2.4618846678672347e-06, 4.140368461013475e-06, 1.3111244791033103e-06, 3.347599469892966e-06, 2.7316784719811237e-06, 1.7181442862681326e-06, 3.476315128317335e-06, 2.999614314956873e-06, 2.4149543577540072e-06, 1.7441586380323302e-06, 3.66648981102996e-06, 3.249401490538054e-06, 3.278117652134433e-06, 1.7733134538443716e-06, 2.1177593450686203e-06, 1.7255294794645081e-06, 2.6623313537336585e-06, 1.7679990808488e-06, 3.391987151458013e-06, 1.4089241105626215e-06, 2.128768672070225e-05, 1.674469256727895e-06, 1.7441586380323302e-06, 2.2984360942719223e-06, 2.336736378838918e-06, 8.986572353871839e-07, 1.644840607982074e-06, 2.1319304615528975e-06, 3.394542680049303e-06, 2.3614678924156404e-05, 2.9655097563944667e-06, 1.8883605247791868e-06, 9.066106145867825e-07, 7.438310447962697e-06, 1.4900228376780057e-06, 1.1037322465074802e-05, 1.0670803672126264e-06, 1.3482520014441584e-06, 2.952825064893474e-06, 5.846157150058399e-06, 2.6921931568495506e-05, 1.0617748619413766e-05, 4.127079424798497e-06, 8.064322105405224e-06, 2.812645114899985e-06, 2.7021207942222964e-06, 1.8337006996743348e-05, 2.993399068161318e-05, 3.447415887139366e-05, 1.3236414400914712e-06, 4.736652862268197e-06, 3.7565346420827433e-06, 1.6389576591064995e-06, 8.982361370620212e-06, 2.157831433167004e-06, 1.4524456681688188e-05, 3.7546483208237636e-06, 3.9266475180232295e-06, 6.92180930137225e-06, 2.1522483890230694e-06, 2.1729590927613145e-06, 3.1809900522377952e-06, 4.704118915274834e-05, 2.9889190085430695e-06, 5.602805110950756e-06, 2.8751148979189473e-06, 2.005423679973017e-06, 4.807109818945244e-06, 3.481208269752858e-06, 2.5037630184552306e-06, 3.346411669616368e-06, 3.2913179618693353e-06, 4.1567579464772695e-06, 3.105982447180238e-06, 6.197707410886335e-06, 4.915620669712237e-06, 3.227655577114359e-06, 4.646543635317401e-06, 2.458767740209138e-06, 1.7701720123742627e-06, 1.9883480989237617e-06, 1.7140595867812297e-06, 1.9883480989237617e-06, 4.6916207893045645e-07, 2.0708802990950762e-06, 1.7828561994526255e-06, 9.716073831668294e-06, 9.270483711253248e-06, 2.5624258031769845e-06, 1.6060858066342182e-06, 8.077822511660393e-06, 3.960800874504366e-06, 1.7511235798728094e-06, 2.4409021349536834e-06, 3.815457303699005e-06, 2.4329348458182695e-06, 9.309819580980906e-07, 5.067513606474283e-06, 3.373984743896949e-07, 1.6645869400918588e-06, 1.6463873923107019e-06, 3.156543569087769e-06, 1.8934002727786449e-06, 1.0936211585665858e-05, 1.612114059392641e-06, 4.6916207893045645e-07, 1.2823522896016187e-06, 3.1288525024578582e-06, 2.8325867552348874e-06, 3.454248386955048e-06, 4.004390722519144e-06, 1.659663650363776e-06, 1.8465919820320036e-06, 1.6128483972538032e-06, 4.6916207893045645e-07, 4.419356303116431e-06, 1.5940495372909991e-06, 6.249465087626322e-06, 1.6155182767536294e-06, 4.0481563722012484e-06, 7.2945724693195495e-06, 2.0944238027663107e-06, 2.976284423371567e-06, 2.5934902118318194e-06, 1.668207836292441e-06, 1.668207836292441e-06, 1.2840337239264096e-06, 4.777054006866875e-06, 3.662330486212381e-06, 5.268670263146318e-06, 5.681579440541997e-06, 5.136422374858353e-06, 6.6742768883277406e-06, 6.607014570268824e-06, 5.5714844750674395e-06, 2.6005315249264727e-06, 6.206302185939645e-06, 1.3238692649227632e-06, 6.321742517393607e-06, 4.499581084131849e-06, 1.5408336120243284e-06, 3.0433355549442484e-06, 2.987024058523423e-06, 5.057100217564281e-06, 2.813008555537697e-06, 3.6785174949949107e-06, 8.430460987121494e-06, 1.1742265212819286e-05, 8.036955871025311e-06, 1.1185491859882968e-06, 2.8588243992596827e-06, 2.3678934929637405e-06, 3.021628261089104e-06, 1.6362489178505718e-06, 9.070803811718035e-07, 7.419056730054663e-07, 9.674419664980537e-07, 2.5124906724946957e-06, 5.412808678024658e-06, 2.6066325405472435e-06, 3.96770310169606e-06, 1.2141632958996236e-05, 4.073677549688717e-06, 4.6916207893045645e-07, 3.1573248078954643e-06, 9.957311875214672e-07, 2.3161650207185263e-06, 1.8155501455740104e-06, 2.0028086154149074e-06, 4.248158154508268e-06, 6.2351491409613635e-06, 1.9127445238845444e-06, 4.248158154508268e-06, 4.001622704004481e-06, 2.100488888224157e-05, 4.248158154508268e-06, 4.960203937387676e-06, 2.5846356192400165e-06, 2.6984001694616897e-06, 4.1484788577492015e-06, 3.1235202365660848e-06, 2.81248440615583e-06, 4.384131364356098e-06, 1.6828262474046948e-06, 6.177132098536138e-07, 2.907549894610678e-06, 1.476568100457895e-06, 1.8364746524369969e-06, 5.1806652286908255e-06, 2.585727622929386e-06, 4.077629069615437e-06, 2.3930004905840115e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.3930004905840115e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.3930004905840115e-06, 2.2262938126788386e-06, 1.377401495770735e-06, 3.183376291431238e-06, 1.3150084947109342e-06, 2.5079986477926037e-06, 3.5735251593389335e-06, 2.9035967084029437e-06, 8.382167231256651e-07, 8.39798991076952e-06, 2.3032887391844524e-06, 6.988563815836896e-06, 7.099624343105483e-06, 3.4365787660653538e-06, 1.1155058047310505e-05, 3.6700470811411775e-06, 2.2565531629350048e-06, 2.108062167699172e-06, 9.966916944052994e-07, 1.6824433094768144e-06, 8.441991836507023e-07, 1.8392234266455443e-06, 5.647044886173917e-06, 3.938377580269984e-06, 1.5509476419218575e-06, 6.1288273018937455e-06, 4.201893435392201e-06, 6.088854089777814e-06, 2.7265712904452853e-06, 4.597403480657345e-06, 4.4200313310036264e-06, 1.4174391066952145e-06, 2.058503758811104e-06, 2.401898638934233e-06, 1.0298848430392564e-06, 3.3208983850641054e-06, 7.684562086504401e-07, 8.16484240737695e-07, 1.6721718509969538e-06, 1.1355912419716714e-06, 2.3766103225124776e-06, 8.932586819174856e-07, 1.2045351586441612e-05, 1.7865173273941797e-06, 4.229507874734487e-06, 4.849653997992987e-06, 4.1484788577492015e-06, 4.1664657064422145e-06, 4.524786762647038e-06, 1.1222893026867206e-05, 2.098946158149234e-06, 3.592462735794293e-06, 1.866018529639758e-06, 5.1689288715491145e-06, 1.8663828835015317e-06, 6.236402944279536e-06, 3.830061430208928e-06, 5.361014975031571e-06, 1.5937614043059963e-05, 3.830061430208928e-06, 9.54401040426345e-06, 5.872982466796787e-06, 2.6873130992369e-06, 3.437647277349317e-06, 5.859543154130939e-06, 1.5099334974988169e-06, 1.2415764065981061e-06, 6.9869536166703635e-06, 4.140627651483785e-06, 5.6372386935451196e-06, 1.2536583414092689e-06, 2.1923689429213625e-05, 1.2326885003735655e-05, 6.875180938977331e-06, 5.158696390192097e-06, 3.346411669616368e-06, 3.346411669616368e-06, 3.346411669616368e-06, 3.346411669616368e-06, 3.346411669616368e-06, 3.346411669616368e-06, 2.153634452555089e-06, 3.347599469892966e-06, 3.718309369616509e-06, 1.7404490588231523e-06, 2.026186702060704e-06, 3.346411669616368e-06, 3.346411669616368e-06, 7.481483133810182e-07, 3.346411669616368e-06, 1.5788300460350383e-06, 2.70279352095343e-06, 1.46564075427987e-06, 1.8230848902894047e-06, 1.648907345192043e-05, 6.4069065109554325e-06, 1.7850483163985783e-06, 3.225896347904771e-06, 5.150310402231859e-06, 1.0937319000290142e-06, 2.1738415827942523e-06, 1.4270893421231965e-05, 1.1885096845669891e-06, 3.2425980687551526e-06, 7.122195210167582e-06, 1.79913601520104e-06, 3.5943751519550763e-06, 4.89549046850669e-06, 2.6581407780163256e-06, 1.7075915859652574e-06, 1.7918097071561322e-06, 2.1749078540257676e-05, 1.7133010611204745e-06, 5.010701074357644e-06, 2.413289189086999e-06, 1.45619272107923e-06, 3.899738770510087e-06, 3.0037379858287995e-06, 4.543963086193281e-06, 5.623223567736697e-07, 3.006249170696297e-06, 9.6499922676714e-07, 2.0529315039200217e-06, 3.3112828929914425e-06, 1.68115590823566e-06, 1.6792082929474533e-06, 1.2157301806855733e-06, 3.2858646491316214e-06, 1.7874017564238642e-06, 1.7874017564238642e-06, 1.5780338048610498e-06, 2.0217765310262343e-06, 4.813447672983435e-06, 2.2537697135164424e-06, 3.5489042861934874e-06, 4.252068506771232e-06, 3.285647089460007e-06, 2.2764801600830378e-06, 1.6549314184349113e-06, 7.251315417597491e-07, 2.343076890566004e-06, 5.860852435007322e-07, 4.520414609577964e-06, 2.221311135564517e-06, 1.8961012113535723e-06, 1.7116954832263294e-06, 4.630880351509379e-06, 1.850467404732725e-06, 2.3500200294765866e-06, 3.147719124051029e-06, 4.647697331920763e-06, 3.107336891359777e-06, 5.354748561172753e-06, 6.456069348405647e-06, 1.148636968943017e-05, 2.3199542271541065e-06, 5.63932082567163e-06, 5.014897678598878e-06, 6.975316331065873e-06, 7.0766103436199215e-06, 3.592462735794293e-06, 4.188349305307399e-06, 3.509301909379991e-06, 1.1885096845669891e-06, 3.4638736853961913e-06, 7.758649365005873e-06, 2.5639972763941475e-06, 3.346411669616368e-06, 3.346411669616368e-06, 4.625859999887876e-06, 2.5650775093772997e-06, 2.5979218451135944e-06, 1.1088351529437289e-06, 2.6478519361553736e-06, 1.911063771966457e-06, 1.409073806414316e-06, 1.8525631973273362e-06, 1.3445203843473528e-06, 3.59113816353669e-06, 4.664328750923063e-06, 2.901462577247363e-06, 2.845765101681164e-06, 3.3379648042763986e-06, 2.5170415043298185e-06, 5.89365175511889e-06, 2.713650881948908e-06, 2.263321845372075e-06, 2.3150127396623425e-06, 3.2066044761621743e-06, 3.10140407447029e-06, 1.6585199808267932e-06, 2.3754020340394066e-06, 5.4326394401393126e-06, 4.127079424798497e-06, 4.518593323285265e-06, 3.34882752432383e-06, 5.5714844750674395e-06, 3.7179926454233375e-06, 4.700099881989904e-06, 2.810184879864786e-06, 2.350841544214854e-06, 2.839318349097078e-06, 1.1287694449770377e-06, 3.936634843503626e-06, 4.6916207893045645e-07, 5.985234027633983e-06, 2.992707815159298e-06, 1.7869598686934927e-06, 3.2722259548501967e-06, 2.992707815159298e-06, 2.585727622929386e-06, 1.6024514120714826e-06, 9.85364709589816e-07, 4.440477476785492e-06, 4.332336621341993e-06, 1.2164238302804956e-05, 2.0830457900654825e-06, 1.43545738711572e-06, 1.8214140943799692e-06, 3.3157333413964474e-06, 4.091609114068383e-06, 4.012470997412411e-06, 2.8628483820139946e-06, 1.014693923953808e-06, 2.955478005807002e-06, 2.7673644114436764e-06, 1.7866293555633434e-06, 2.9424976052873643e-06, 4.714023531953415e-06, 8.067512350062005e-06, 3.337706944592089e-06, 1.3231029394646453e-06, 1.8786071185942937e-06, 3.4548402356361043e-06, 4.169183208693726e-06, 4.04470029902868e-06, 5.296276358141208e-06, 4.453045335626011e-06, 3.1014037907026486e-06, 7.690100608834885e-07, 1.0860516635606111e-05, 3.081938890350676e-06, 1.2501569647384616e-05, 5.011490607543234e-06, 3.232710868780312e-06, 1.1133727909306247e-05, 4.091609114068383e-06, 2.078813914351986e-06, 3.1320036830344613e-06, 2.5683885792976106e-06, 2.0893216487624387e-06, 4.807109818945244e-06, 3.891096628935303e-06, 1.9957714411071277e-06, 1.8776043742196325e-06, 1.565047846925768e-06, 1.3445203843473528e-06, 9.720935286790265e-07, 4.170701019111099e-06, 2.3723777772354266e-06, 2.6704097549947934e-05, 7.225692152715406e-06, 1.8165367579790817e-06, 1.8456403533024862e-06, 4.105466108270867e-06, 8.211551370664185e-06, 2.177353966657511e-06, 9.344076620194846e-07, 4.433199107212212e-06, 4.827194077619394e-06, 1.8905995041331689e-06, 1.0108097011799643e-06, 1.159932384989829e-06, 2.780633128531029e-06, 1.7276091156503209e-06, 2.455246167792247e-06, 4.394639374367129e-06, 2.665962404280764e-06, 2.8570457006332974e-06, 1.6031085452274533e-06, 9.076497650507349e-07, 2.0368306785282045e-06, 1.8554856423772645e-06, 2.0049066432915116e-06, 3.592462735794293e-06, 2.512513881320564e-06, 1.7701720123742627e-06, 7.171419023811106e-07, 2.082311070617085e-06, 1.4096650530934548e-06, 7.171419023811106e-07, 8.038944635257461e-07, 7.3027959199162075e-06, 8.247957055396285e-07, 1.059988786688589e-06, 2.51337249989927e-06, 2.111224366973584e-06, 6.800567691772618e-06, 4.907520089204796e-06, 2.51337249989927e-06, 6.686385393592756e-06, 4.646477455268053e-06, 2.7879703761608244e-06, 2.51337249989927e-06, 4.301337495097183e-06, 2.51337249989927e-06, 3.3100697675808583e-06, 4.917906367433944e-06, 6.376260799306799e-06, 4.202616856494381e-06, 4.5059853139908245e-06, 2.659775655851495e-06, 3.598401236711556e-06, 2.378102145764561e-06, 3.4520699944044114e-06, 3.086765954334156e-06, 3.6671208863327804e-06, 3.1084454260835687e-06, 9.11683314409125e-07, 1.4512642218003213e-06, 1.5429607576283394e-05, 4.795584593584684e-06, 6.71668089664516e-06, 3.746738246344581e-06, 6.956852512695364e-06, 4.644599227754646e-06, 2.782819355310105e-06, 5.029583770969965e-06, 1.2976711914334296e-06, 8.969110436922176e-07, 1.7897983655068007e-06, 1.3504742349473875e-06, 2.3498408541128673e-06, 2.17379243321668e-06, 9.976697716679074e-07, 2.9983252059994637e-06, 2.879951746759389e-06, 1.8386026810866876e-06, 2.17379243321668e-06, 6.305860241166182e-06, 1.4293044431978715e-06, 3.905073095017059e-06, 1.2466332898087694e-05, 4.912517740987807e-06, 4.610957437993659e-06, 3.0090588511995915e-06, 3.173847917870303e-06, 4.519889767412154e-06, 9.893555039862305e-06, 2.760387907359893e-06, 2.0185684783027874e-06, 2.0235463215036043e-06, 2.150827925672278e-06, 1.0634166367771562e-06, 3.373322939219227e-06, 7.135282589086653e-06, 2.0180686944460555e-06, 1.0773437961256517e-06, 3.0077559596549918e-06, 4.827194077619394e-06, 2.843350372513517e-06, 2.3951198543415135e-06, 8.504431831113731e-06, 2.722955434907511e-06, 1.2012617199810992e-05, 8.291342424066121e-07, 2.692093567607768e-06, 3.347599469892966e-06, 2.992707815159298e-06, 3.3381178701666953e-06, 1.7441586380323302e-06, 3.6471477091530745e-06, 3.1511698297746385e-06, 4.962399805207265e-06, 3.818091190554478e-06, 1.8858499396553344e-06, 8.908615062198784e-06, 4.277002755703258e-06, 2.6777947067990527e-05, 2.9365465072895438e-05, 1.7363840415786013e-06, 9.109410859527607e-06, 5.607638372114081e-06, 3.3204439484819878e-06, 1.7310657708683468e-06, 2.288973914092243e-06, 3.010617878206291e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 4.171828113337423e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 0.00022694006987183005, 4.031913888581133e-06, 1.0812126599329047e-06, 8.080559465491839e-07, 1.2958766489403733e-05, 4.026821346578329e-06, 2.1452243343700037e-06, 2.089790280158887e-06, 2.7798191093696844e-06, 3.725512508902553e-06, 4.676717468208723e-06, 2.9374455711910934e-06, 3.0462991039939396e-06, 1.4743574583419873e-06, 1.098432276421449e-05, 1.4981448943190037e-06, 7.6356117711459e-06, 4.212786105865247e-06, 1.9086994967612458e-06, 3.6639234057973706e-05, 1.4043144825400185e-06, 2.5548249171161186e-06, 3.6340187273177323e-06, 3.90679858278566e-06, 3.3953299241553857e-06, 3.3138214988008855e-06, 2.2053239536030915e-06, 2.5176323328682577e-06, 2.741157656592836e-06, 3.1069584971466453e-06, 1.7171678540875854e-06, 4.077534430513048e-06, 5.995242652526407e-06, 2.5842348080063297e-06, 1.1099922704836578e-06, 2.51337249989927e-06, 5.363703339486306e-06, 2.426351098877216e-06, 5.067746832727813e-06, 6.940254267938244e-06, 2.51337249989927e-06, 4.210197873431542e-06, 2.5067801738599606e-06, 1.5509687388016484e-06, 6.70156058238272e-06, 2.2429554827324603e-06, 6.661021364917677e-06, 1.3174852582832123e-05, 2.51337249989927e-06, 6.783148739470499e-06, 2.7122184636894706e-06, 5.497323393150349e-06, 2.51337249989927e-06, 5.6140870378816915e-06, 8.941996539858293e-06, 3.6557554235733036e-06, 6.528119029625054e-06, 2.51337249989927e-06, 3.924226022668915e-06, 2.274348616255275e-06, 4.248158154508268e-06, 2.542555591340652e-06, 4.248158154508268e-06, 1.3322649223404477e-05, 4.248158154508268e-06, 2.0014662705359974e-06, 5.694996420719178e-06, 2.4149543577540072e-06, 4.248158154508268e-06, 1.6583709196293253e-06, 1.5432456604535353e-06, 2.7461118753491837e-06, 1.2431235023289693e-06, 1.9909153488271913e-06, 1.8047345281285832e-06, 4.6896337043265536e-07, 2.7860115516298005e-06, 4.469086749203598e-06, 2.295491309427358e-06, 4.685351746035572e-06, 1.8999975555994731e-06, 4.058383450895962e-06, 4.469086749203598e-06, 1.7151107253906973e-06, 3.0059099857524333e-06, 2.51337249989927e-06, 4.921364261855467e-06, 2.485973382869108e-06, 2.187059699396993e-06, 1.2318381001813611e-05, 5.393157629552722e-06, 4.612999258170755e-06, 8.616145799889319e-07, 6.599269585531369e-05, 1.5065542700985494e-05, 2.181593212714989e-06, 1.7391347812630625e-05, 1.6729333354558983e-06, 1.6391346618298343e-05, 3.1549819600164054e-06, 5.336183156502319e-06, 2.8875016573001157e-05, 3.451478927319473e-06, 1.374300012981805e-06, 1.1241469311376395e-06, 3.3623233442191164e-06, 3.1101598596803886e-06, 1.702547905179189e-06, 4.6175638933640106e-06, 1.8239801905499975e-06, 2.3930004905840115e-06, 5.457875504844355e-06, 7.587292944451419e-06, 7.587292944451419e-06, 7.047046497112655e-06, 1.5213992434234634e-05, 4.499257159679458e-06, 5.296276358141208e-06, 1.7701720123742627e-06, 3.2231625704817617e-06, 4.4947556461706034e-06, 6.5256139193824185e-06, 1.8858499396553344e-06, 2.4735278546523498e-06, 2.4704915357992033e-06, 2.141721430645982e-06, 1.1738287113272439e-05, 1.2978522709557136e-06, 2.8074756910203913e-06, 4.927943455894515e-06, 1.906860761249636e-06, 1.3317332366723796e-06, 2.4291696894829697e-06, 4.248158154508268e-06, 2.496945490441163e-06, 1.9127445238845444e-06, 4.248158154508268e-06, 4.4982258478996776e-06, 1.947238856512008e-06, 1.8079773932924012e-06, 2.809487308461313e-06, 2.186213204903544e-06, 2.45179140399835e-06, 4.4452465413419815e-06, 3.304182920961325e-06, 1.4858175365230256e-06, 3.862221105556474e-06, 6.011393750412903e-06, 4.61749138978574e-06, 1.4575774885143136e-05, 5.654894905887575e-06, 7.199759080393244e-06, 4.483154555515716e-06, 3.0762768240281595e-06, 6.562558576150998e-06, 2.959557564233408e-06, 7.303408666999834e-06, 6.1157498260154675e-06, 1.4525347392182048e-06, 2.9287534242183294e-06, 1.1679752377492025e-06, 5.464981844559192e-06, 1.9638211512535317e-06, 8.236251045548531e-06, 1.8786071185942937e-06, 3.571195597957986e-06, 1.1307463058471174e-06, 1.3637549478020658e-06, 3.479286647877834e-06, 8.651046528297575e-06, 5.8687432624124515e-06, 4.615909961392969e-06, 1.0816810166917076e-06, 1.3457901698130592e-06, 1.7431751837782426e-06, 2.670707735926461e-06, 5.968939701214383e-06, 2.182672994083695e-06, 7.341768708774778e-07, 2.524439782269443e-06, 9.681498790442828e-06, 2.6876735421740932e-05, 1.287393707686287e-06, 1.1510570619779515e-06, 1.287393707686287e-06, 1.7701720123742627e-06, 2.1090314212494093e-06, 3.0609998018870985e-06, 3.212136083709667e-06, 2.5087347908585297e-06, 1.4858175365230256e-06, 1.798666595416959e-06, 1.5532145472283166e-06, 2.3447869714952096e-06, 4.299719426001789e-06, 1.8978342380997948e-06, 2.216051062597561e-06, 2.51337249989927e-06, 2.456432475270844e-06, 2.51337249989927e-06, 4.642134308137218e-06, 8.075094858408095e-06, 3.597646585372654e-06, 7.555424049562647e-06, 9.963301258738358e-06, 2.6157981075247386e-06, 9.74579847854732e-07, 1.3457901698130592e-06, 1.3943622260955471e-06, 7.022828014258039e-06, 1.9217886257313046e-06, 5.5441625196406105e-06, 4.0009806776178855e-06, 1.2118863502078515e-06, 1.6092023806022482e-06, 1.6056925152894538e-06, 2.543486999666943e-06, 3.2154756322355942e-06, 4.442467041661535e-06, 8.960937009808451e-07, 5.730445154095507e-06, 2.705532476498647e-06, 3.577090815605212e-06, 2.132704291592586e-06, 8.181958940296495e-06, 2.51337249989927e-06, 1.2210479495846425e-06, 2.468347108527062e-06, 4.828251861585145e-06, 5.0464969527562664e-06, 2.3449813973345426e-06, 2.8181610495623007e-06, 2.51337249989927e-06, 2.51337249989927e-06, 4.8432729764615585e-06, 1.0282980096375934e-05, 1.079880141112529e-05, 5.345828950103731e-06, 4.248158154508268e-06, 2.8030580223487377e-06, 4.248158154508268e-06, 2.4937907141495027e-06, 2.499486630537091e-06, 2.716114701425202e-06, 2.8888266342465836e-06, 2.4735278546523498e-06, 9.507278997422225e-06, 2.329402168427181e-06, 7.504817660437006e-06, 6.866602157805142e-06, 4.607776526940825e-06, 1.1642435853154103e-06, 3.68775738503928e-06, 7.032940467694928e-06, 3.4111790231718434e-06, 4.4947556461706034e-06, 4.746757658912044e-05, 1.4820720647572233e-05, 1.785417116685109e-05, 2.4876893864481234e-06, 4.582469044132215e-06, 7.180523566029972e-06, 3.976995943582235e-06, 1.0637526954424474e-05, 1.616233560570536e-06, 1.2783629261879955e-05, 1.746833886416883e-06, 1.2414649322250138e-06, 2.1851469003726804e-06, 2.3574397190728087e-06, 4.99934983281166e-06, 2.272530731910421e-06, 5.474572513246704e-06, 5.406441407447752e-06, 5.23506340628299e-06, 2.165632133601311e-06, 8.851913749635715e-06, 3.623360416443577e-06, 2.51337249989927e-06, 3.599862156717589e-06, 2.51337249989927e-06, 1.2766565156371921e-05, 2.314332637946946e-06, 4.130216055310592e-06, 4.447082739516886e-06, 1.864277565884778e-05, 4.182834644236531e-06, 1.8637269493306277e-06, 2.51337249989927e-06, 6.383420869707199e-06, 3.1182301535076413e-06, 3.254780192053417e-06, 1.01417045152195e-05, 4.8058749976302466e-06, 6.492108788005228e-06, 1.0445577965595569e-06, 1.101110449472316e-06, 2.239861664468425e-06, 1.101110449472316e-06, 1.3457901698130592e-06, 6.052426080975757e-06, 9.285890812371862e-06, 1.0488889810401227e-05, 1.295034782053458e-05, 2.378102145764561e-06, 3.125653355984874e-05, 2.3162371456280554e-06, 4.691026649948051e-06, 3.4791814902983924e-06, 3.679889803393607e-06, 5.484090696193966e-06, 3.6770740734409656e-06, 3.3876031192298195e-06, 2.3405728741128703e-06, 1.4469579237908834e-06, 5.136719479485255e-06, 8.842454226594659e-07, 2.6178623204523495e-06, 3.0572412272504133e-06, 3.972653573801824e-06, 3.223405583941253e-06, 2.7514208406335495e-06, 1.0735173259310583e-05, 2.6346234357485553e-06, 2.6239965525504803e-06, 2.51337249989927e-06, 2.141752980444555e-05, 5.15493749819631e-06, 1.319614541426735e-06, 1.2917853220539344e-06, 7.202524195605084e-06, 1.934044398099013e-06, 2.51337249989927e-06, 5.0802765100371e-06, 2.7549477306674518e-05, 2.7080924128778874e-06, 6.776071977254505e-06, 5.481401840700376e-06, 4.017603838621908e-06, 5.64954481647878e-06, 7.379487161297211e-06, 3.7892673937586506e-06, 5.618694401449808e-06, 2.648085211305257e-06, 2.619477538218924e-06, 2.87672095123692e-06, 8.145904269751114e-06, 9.383153817168915e-06, 1.7271628884056665e-06, 4.248158154508268e-06, 4.373033123047146e-06, 3.1865822391081472e-06, 5.614964339567988e-06, 2.51337249989927e-06, 5.6919593792823856e-06, 2.1269935523849532e-06, 1.0197314762771255e-05, 6.6509741768654795e-06, 2.3158386853418925e-06, 4.287461446662802e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.3930004905840115e-06, 1.7664405620649212e-06, 0, 7.037566103847054e-06, 1.26698363117262e-06, 4.204724580649436e-06, 7.037566103847054e-06, 2.1649482024443395e-06, 4.204724580649436e-06, 2.2801291978481027e-06, 1.6579467729689092e-06, 3.9133626291668184e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 6.691587758275572e-06, 2.774013045040797e-06, 2.007720567417332e-06, 2.44227993629147e-06, 1.1930268616092244e-06, 4.303015426186096e-06, 3.177026537666802e-06, 2.3621549744306467e-06, 8.918176778349468e-06, 3.976788354745282e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 3.2641762975824147e-06, 4.0481563722012484e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 1.3653365623189898e-06, 3.2663004165067034e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 1.3919013971110958e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.0481563722012484e-06, 7.037566103847054e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.204724580649436e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 7.037566103847054e-06, 4.0481563722012484e-06, 4.204724580649436e-06, 1.3323934740677448e-05, 4.0481563722012484e-06, 4.0481563722012484e-06, 7.037566103847054e-06, 6.104921008632225e-06, 4.95665132985891e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.3930004905840115e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.3930004905840115e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.39536425013163e-06, 2.3930004905840115e-06, 1.377401495770735e-06, 3.356640940122911e-06, 2.3930004905840115e-06, 2.3930004905840115e-06, 2.2262938126788386e-06, 2.3930004905840115e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 2.3930004905840115e-06, 2.2262938126788386e-06, 2.092185199657457e-06, 3.9023263010722616e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 2.7306709868805843e-06, 4.778296433340026e-06, 2.3930004905840115e-06, 1.2737275180776747e-06, 1.1656015130105055e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 7.037566103847054e-06, 3.847232215549591e-06, 1.7558502389942294e-06, 3.116849392912977e-06, 2.2262938126788386e-06, 2.2262938126788386e-06, 8.016573374732749e-07, 1.5816169922973863e-06, 1.7740574811945104e-06, 2.0506287678149454e-06, 3.269289088685115e-06, 4.204724580649436e-06, 7.037566103847054e-06, 1.668207836292441e-06, 0, 3.1304012738074888e-06, 5.71346845193851e-06, 2.3930004905840115e-06, 2.9894535151320155e-05, 1.50719942837218e-06, 4.73107893258553e-06, 9.61485150368252e-07, 2.5738481820061045e-06, 1.817002418234169e-06, 8.029843127788567e-07, 2.51337249989927e-06, 2.51337249989927e-06, 2.51337249989927e-06, 5.224964686634436e-06, 2.8514843062000863e-06, 3.7226743685015173e-06, 3.4196095364337437e-06, 5.079528128208797e-06, 5.278345341284997e-06, 2.4617713677840496e-06, 4.036964360253127e-06, 9.898452014074546e-07, 1.6617150396940981e-06, 3.0307118030575175e-06, 3.1337530445391834e-06, 4.190848001803076e-06, 1.3567231358158636e-06, 8.318382482906625e-06, 2.440175142084099e-06, 2.141721430645982e-06, 1.2354097781500584e-06, 1.1604080499760613e-06, 1.766433666157384e-06, 1.1857899614797604e-06, 1.9035061934322614e-06, 1.9035061934322614e-06, 2.084475430216412e-06, 1.2714416104018573e-05, 1.7740829115210534e-06, 4.036441984360696e-06, 2.0134972996459066e-06, 2.319013010806536e-06, 3.260611986870681e-06, 3.244567180728221e-06, 2.0280167614219364e-06, 2.7514208406335495e-06, 3.244567180728221e-06, 3.244567180728221e-06, 2.7569855023174004e-06, 4.252046979423764e-06, 5.462170500699809e-06, 4.719939010970408e-06, 1.7608815179914986e-05, 2.8979201366412683e-06, 2.51337249989927e-06, 3.6269892245804476e-06, 5.791589790445068e-06, 2.2346294624444277e-06, 1.1241469311376395e-06, 8.423606918546817e-06, 2.51337249989927e-06, 2.51337249989927e-06, 3.0489658176823904e-06, 3.5594086058705766e-06, 4.0702517015310006e-06, 6.324406103972376e-05, 1.1799528618170924e-06, 2.5813109652094675e-06, 2.51337249989927e-06, 4.343995089189446e-06, 6.337331618267403e-06, 4.799596810986736e-06, 7.599852229833846e-06, 2.057566449103812e-06, 6.110376758395723e-06, 1.7918790296274219e-06, 2.51337249989927e-06, 2.51337249989927e-06, 1.0044761157080143e-05, 7.947136610806579e-06, 5.773948694761286e-06, 9.222610545616984e-06, 7.46794240066927e-06, 6.386885971425013e-05, 4.342222843688241e-05, 2.7823784762810795e-06, 3.481208269752858e-06, 1.4512629824684634e-06, 2.341122315633095e-06, 1.8095771405108467e-06, 7.109482984904062e-06, 5.147692177045339e-06, 1.5132271214795566e-06, 4.014841686447663e-06, 2.075618437601361e-06, 2.7316784719811237e-06, 3.6563241732145043e-06, 4.6347634391562345e-06, 1.6828262474046948e-06, 2.2341527107246302e-06, 3.5644655070543836e-06, 3.966859507367193e-06, 2.51337249989927e-06, 5.119724158309137e-06, 1.35441264850174e-06, 2.51337249989927e-06, 2.51337249989927e-06, 7.734031895864482e-06, 2.51337249989927e-06, 4.056475645244297e-06, 6.423495013786964e-06, 1.0524741208647097e-05, 3.534510265669573e-06, 4.925295416466445e-06, 1.3988676156645022e-06, 1.2085404332304148e-05, 1.2068742992606581e-05, 2.4735278546523498e-06, 2.681613021153287e-06, 1.5554248971734024e-06, 1.5554248971734024e-06, 3.1447828481826677e-06, 1.012753826825899e-05, 4.4947556461706034e-06, 2.3590446591682424e-06, 5.249813249540969e-06, 2.3632316138727043e-06, 2.455246167792247e-06, 4.307574846826484e-06, 2.8628483820139946e-06, 5.679065334877873e-06, 2.51337249989927e-06, 3.1511698297746385e-06, 1.328506895024136e-05, 6.6742768883277406e-06, 1.971392508273115e-06, 3.3157333413964474e-06, 2.2341527107246302e-06, 7.180874938054777e-06, 3.2406081115959383e-06, 2.51337249989927e-06, 1.4349939599262509e-06, 5.11845750550653e-06, 4.600512126403871e-06, 3.7546483208237636e-06, 4.318967641101614e-06, 2.51337249989927e-06, 3.4923853608932967e-06, 6.092944658050585e-06, 8.426382948191443e-06, 6.4846910811077876e-06, 4.707256649294883e-05, 9.28930920606764e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 2.8709146171669645e-06, 1.159132159700182e-06, 2.7666132565794465e-06, 1.258391624254789e-06, 4.204724580649436e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 7.037566103847054e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.204724580649436e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 7.037566103847054e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 7.037566103847054e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.204724580649436e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 7.037566103847054e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.204724580649436e-06, 4.0481563722012484e-06, 7.037566103847054e-06, 4.0481563722012484e-06, 4.650965519778906e-06, 6.109136010419623e-06, 6.31596327638147e-06, 3.6136483405912854e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 1.6535731794568116e-06, 2.3016997242364253e-06, 1.668207836292441e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 2.6729628310743653e-06, 2.1649482024443395e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 7.037566103847054e-06, 4.0481563722012484e-06, 1.929028218083888e-06, 7.2889286584159535e-06, 4.204724580649436e-06, 7.037566103847054e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 7.037566103847054e-06, 4.204724580649436e-06, 4.204724580649436e-06, 4.204724580649436e-06, 2.1649482024443395e-06, 1.7086634070199624e-06, 4.204724580649436e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.204724580649436e-06, 4.0481563722012484e-06, 9.949691655723675e-07, 1.668207836292441e-06, 2.1649482024443395e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 1.4023163673611713e-06, 4.204724580649436e-06, 4.204724580649436e-06, 4.204724580649436e-06, 4.204724580649436e-06, 4.787237034795636e-06, 2.6729628310743653e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 1.9368169608604516e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.0481563722012484e-06, 4.204724580649436e-06, 2.2843689293712566e-06, 2.7256787018854346e-06, 2.7667485607914795e-06, 3.7206067958442e-06, 5.913739576822696e-06, 5.663483933132959e-06, 2.7667485607914795e-06, 2.397103902374679e-06, 4.719362371879303e-06, 3.6151495203075424e-06, 2.3930004905840115e-06, 0, 1.8540924350779268e-06, 1.0508418897940866e-05, 2.2110902308521357e-06, 2.5830932505610206e-06, 1.621759862198055e-06, 2.1582234082542646e-06, 1.218998722679985e-05, 1.555690623749971e-06, 1.170943777791593e-06, 1.7872900613941927e-06, 6.124599087675123e-06, 1.5944935178043031e-06, 4.101075959711408e-06, 2.5268979622632645e-06, 1.2452703283675847e-06, 1.4398809505625206e-06, 2.5984530021916477e-06, 1.3032481280233628e-06, 1.5677278881489783e-06, 1.2548969296174451e-06, 2.068603264524075e-06, 1.2260844781468447e-05, 1.7542952511407465e-05, 2.92071233518647e-06, 6.635164138120714e-06, 1.807538632808358e-06, 8.387771870918426e-06, 6.830107338883297e-07, 2.7860115516298005e-06, 1.8151468041400333e-06, 1.2143128160254834e-05, 2.19629298992266e-06, 3.036665175086965e-06, 1.512401785611209e-06, 1.657912597398716e-06, 1.6478627556500466e-06, 2.17379243321668e-06, 2.0009045762201513e-06, 4.864553506765525e-06, 9.39039064055172e-06, 2.6630247126142683e-06, 1.1722588757702574e-06, 2.4561946959888278e-06, 2.7576676830170076e-06, 2.17379243321668e-06, 2.17379243321668e-06, 1.946017528737314e-06, 2.17379243321668e-06, 2.17379243321668e-06, 8.842454226594659e-07, 3.4036675636594515e-06, 1.7987849332320027e-06, 1.0323867226289342e-06, 1.8622952641235126e-06, 1.1241469311376395e-06, 1.4099869795214557e-06, 3.3497887899008407e-06, 2.5828962653316627e-06, 2.416596942776661e-06, 2.17379243321668e-06, 2.17379243321668e-06, 2.17379243321668e-06, 2.17379243321668e-06, 1.3488048925247135e-06, 1.4099869795214557e-06, 9.243549897847433e-07, 3.2762558763637565e-06, 1.502176545729482e-06, 1.8659211283293692e-06, 2.5171410728887624e-06, 2.273957153957675e-06, 2.0763646720498304e-06, 2.5433616700056185e-06, 1.4232404704577345e-06, 8.015906416941515e-07, 2.8259826475659717e-06, 2.17379243321668e-06, 5.140918614312751e-06, 9.055936444428471e-07, 4.398122733301555e-06, 4.1001439214909806e-05, 1.1948789954595814e-06, 3.628911240184607e-06, 5.602805110950756e-06, 1.9178836842953417e-06, 1.812726010478459e-05, 5.23506340628299e-06, 2.3951198543415135e-06, 2.8992188650743422e-06, 1.646067341287344e-05, 3.9586960139203866e-06, 2.9510787317015894e-06, 3.3341429285943192e-06, 2.17379243321668e-06, 2.17379243321668e-06, 2.17379243321668e-06, 1.3912036078147612e-06, 2.17379243321668e-06, 2.0014012004022417e-06, 2.0590129835674717e-06, 1.0844303625897795e-06, 5.3766994736694896e-06, 1.8562924537949243e-06, 2.17379243321668e-06, 2.17379243321668e-06, 3.3095001877187326e-06, 5.818698405071819e-06, 2.6695557028138473e-06, 2.2806624332510458e-06, 2.4613785369408354e-06, 2.131633204439011e-06, 4.692127637839912e-06, 2.018587370840939e-06, 2.392890105235513e-06, 2.17379243321668e-06, 2.17379243321668e-06, 2.17379243321668e-06, 2.17379243321668e-06, 1.1340960292297039e-06, 1.2031862551064926e-06, 1.5790104355304356e-06, 4.6661233311692475e-06, 4.400886926146525e-06, 2.1037598033800244e-06, 1.566152543526398e-06, 1.4188192120016533e-06, 4.967615340302785e-06, 4.727988717953323e-06, 1.9923440705186272e-06, 9.934312391057377e-06, 1.6282681842052743e-06, 2.9722103056826045e-06, 2.17379243321668e-06, 6.073258766711686e-06, 4.763771555909344e-06, 1.4431697394813947e-06, 1.4640548551966131e-05, 1.3817200420949715e-05, 3.949556149036165e-06, 3.501861082112008e-06, 2.2277521235653205e-06, 8.309484370232033e-07, 1.6535731794568116e-06, 2.6384432887859927e-06, 1.5270168333179234e-06, 5.481401840700376e-06, 2.8244961890236854e-06, 2.471039068629159e-06, 3.2923570228387403e-06, 2.575103771685661e-06, 3.592462735794293e-06, 3.2923570228387403e-06, 1.9344050137734774e-06, 3.1831153261582202e-06, 4.749525236442172e-06, 6.892769797309318e-06, 6.774551199409289e-06, 7.802101029202839e-06, 3.8324619180823116e-06, 6.5009891302660135e-06, 5.510168457497078e-06, 3.696540217543537e-06, 1.0167591034372518e-05, 2.3851468184350993e-06, 5.362672596860187e-06, 7.907816565184607e-06, 4.588735550205233e-06, 1.1891925740516952e-05, 1.6738653172186387e-05, 1.8786071185942937e-06, 3.9563698002089135e-06, 4.0918862285350955e-06, 2.6497391688529393e-06, 7.4894334507875075e-06, 4.4711224734041516e-06, 2.5454523047927625e-06, 3.262008541611742e-06, 2.735833324416299e-06, 3.8189843296584135e-06, 1.287393707686287e-06, 5.552714153368537e-06, 2.141721430645982e-06, 1.3457901698130592e-06, 1.1307463058471174e-06, 1.3025534465269889e-06, 2.4235733647916623e-06, 9.090382194057628e-06, 2.089790479334439e-06, 5.204671689292294e-06, 4.303015426186096e-06, 2.603512155760232e-06, 6.083007626336799e-06, 6.349783996970762e-06, 1.3094886609568657e-05, 5.302926692870902e-06, 4.852721191036552e-06, 2.2510405695994e-06, 4.127079424798497e-06, 3.232710868780312e-06, 2.8628483820139946e-06, 5.919505137540492e-06, 2.8628483820139946e-06, 1.4305092266751277e-06, 5.688934367789784e-06, 1.9086994967612458e-06, 5.400073556799129e-06, 2.261661348532712e-06, 7.4590248640161195e-06, 1.850467404732725e-06, 1.0422428805825745e-05, 1.3986271616246643e-06, 1.5106214290688497e-06, 2.586496757021944e-06, 1.4248195005583767e-06, 3.7215274690239718e-06, 5.521839359385822e-06, 1.5605846611443094e-05, 1.1381960334204067e-05, 1.2848146856913188e-06, 7.974726701198498e-06, 4.5364254326108074e-05, 4.787900150994305e-06, 1.83724529233567e-06, 4.4947556461706034e-06, 4.306508730975103e-06, 2.089273061393581e-06, 1.850467404732725e-06, 3.1904775912153487e-06, 3.0686799063340174e-06, 2.089273061393581e-06, 2.689435592567377e-06, 2.9864016318896967e-06, 3.8261989906101075e-06, 4.6755381727054035e-06, 7.997847100781376e-06, 7.170656460833182e-06, 4.08029439568193e-06, 4.6623540783693615e-06, 7.549092733129613e-06, 1.4707644623073802e-05, 2.9106632323486273e-06, 1.3219900253784667e-06, 4.024174360730916e-06, 1.478646511953805e-06, 2.836535265860968e-06, 1.0321229249156715e-05, 2.7118674248131146e-06, 4.91238951650961e-06, 5.607638372114081e-06, 8.842454226594659e-07, 1.951119727511889e-06, 1.4413892436191609e-06, 2.978299018312262e-06, 1.1510570619779515e-06, 1.5151780089992951e-06, 1.4867288938599167e-06, 1.3741465304631737e-05, 1.2027767662452647e-05, 1.9058876353371737e-06, 1.1510570619779515e-06, 1.505191834559408e-06, 1.0445577965595569e-06]
All_score_list=  [-4.391300366888448, 1.5107412421843138, 0.3273712415026203, 2.3314888015124122, 1.36582239662168, 1.6013243458091104, 0.5633564316567753, 1.8118267045549679, -0.6794233443830591, -4.634924438573464, 1.5733047467527932, 1.0048462921361068, 0.9572850958471456, -1.300209997313539, 1.1605003630862405, 0.8238729654049642, -0.10701840856151748, -0.15823618961371946, 0.8138249062141766, 0.26582401857701166, 0.6490955791034547, 0.5861397181356862, 0.00208250959800943, 0.5606013398740595, 0.451825908463351, 0.6510251862504856, 1.5174526466293743, 1.4880312223586505, 0.7522961026327664, 1.5660985859109409, 2.1947671279789374, -0.14321376631866892, 0.4164617156520207, 0.49104635746547953, 1.136941394744725, 0.8936722676744272, 0.7484947731032883, 0.33774994378467393, 1.731970459026543, 1.0927818664072069, -0.22164143986005744, 0.4527178136039878, 0.15388478335630904, 0.8127319477186243, 0.4097637291303543, 1.5943602323074852, 0.6351246194257831, -0.7202720795506061, -1.9531350054176764, 0.9976273688584592, 1.05803941651989, 2.093723346643656, -2.4284250015108997, 1.4069283972724576, 0.5942932049958225, 0.5508389541252939, 1.9226404066256226, -1.4029877564222344, -0.4209872751082059, 0.7342311940410237, 1.4494091574547137, 1.3762373066578375, 1.0768353220937799, 0.6701870757986846, 0.20342640102800386, 1.4206346314653515, 1.0452599250482846, 0.6909604469354623, -1.0596455588173612, 1.1483542764462218, -0.6570066025080564, -1.4514743797706047, -0.8016096391823392, 0.8919424669320907, 0.8445957999235468, -0.4654292871271838, 0.3570794858493106, 0.6822880335319829, 1.9299140680770879, 1.099773323974478, 1.1044117991943876, 0.9797496746150326, 1.7943539109027966, 0.46092372987487584, 0.6166086793712549, -0.6306928622797358, -1.608367007910398, 0.5055835076892911, 1.3608422935771343, 1.4257272538501866, -0.1237592896265883, 0.6063895952712557, -2.209021633215967, -0.31971718613951716, 1.115647335102963, -1.356538297597642, -4.4513356568876254, -0.675099096278295, -0.0006849574499234734, 0.2619629142057748, -2.755318993575025, -1.0689445100700874, -1.0190684967498456, 1.7830579225719794, -0.18258148345099737, 1.7840943331947208, -1.5388986481013431, 0.12218632489525547, 1.4919703719974966, -4.55120101320109, 0.2835804502070777, 0.6772487177365701, -0.6120388727365889, 0.8184414499764431, 0.41811863426873686, -0.6096830305958, -1.127926849038258, 1.1788720532350474, 0.9679786582499719, 0.18384283708398405, 0.39810492234036043, 1.5159448617527298, 0.050973906388059986, 0.30356649045426387, 1.1849811656114377, 0.01481724453516739, 1.567858552645667, 1.292474963205158, 0.9676164258947115, 0.5555739690263664, 1.4570082310842092, -0.1988274231087038, -2.40349491182477, -0.045274645102149846, -0.10105803872819377, -1.5444544999682317, -1.4395984037267597, -0.03162922918013464, -1.1124968038271665, 0.2936237479008893, 1.4256515369959697, 1.3721233206615595, 1.2654657320519642, -2.6003588845205385, 0.11494905653900475, -4.359146509055597, 0.9304892523026164, 1.2800207934130408, 0.7857580650429196, 0.8244144580830738, 0.5442483545228611, 0.9084685787147742, 1.9007444481935478, 1.3195915899843287, -0.2710450797972075, -8.543148096737816, 1.1214351909985343, 1.5377724693740464, 1.8604616968778003, -1.4237549507005915, 2.1383956455866877, 1.210725113526335, -0.9978462230126209, -1.2189511329520828, 0.9364893691108488, 0.38974566151128354, 0.2919882945564096, 0.6252656239695495, 0.5888155223281915, -1.0790854544532082, 0.8915980525085428, -0.35752097956079665, 1.1095539656670503, 0.9110866534328727, 0.7266888096785246, 0.36021636752636343, 0.7537460582397845, 1.3100693387811921, -0.838528041155966, -0.8959166131858393, 1.6089184077412086, 1.8133049304825568, 1.8433399314231258, -0.4271683397396935, -0.2670086552420362, 0.2575323388722345, 0.36462693045711403, 1.8294908973727482, 0.6481771736668914, 0.9472658780989492, 0.8654614801691631, 1.3374116220492678, -0.21424276969555653, -0.9391743671345979, 1.027883953227139, -6.69992586565816, -3.5381215433434696, -2.6902512455741, 1.1373414499490069, 1.2146877614099816, 0.7703056549871498, 1.8483825806305607, 1.5660412435031077, -0.30270381835339066, -0.1088399137409887, 0.5483021922470604, 1.273033888685292, 1.166092959935656, 1.828510635866828, 1.9505425137058674, 1.4449239729884493, -0.8541197760173823, 1.2627358363322685, 0.9329567559872528, 0.8207441383149079, 0.41485164250324996, 0.509443202556974, 1.1048920097314152, 1.7822238291384433, -4.4185711836818395, 0.7475600491848003, -1.084889744361013, 0.26962812068986947, 1.7047411212141814, -0.8273178197241836, 1.2231745745713407, -0.33778995823392843, 0.5735397778276289, 1.3028956626994799, 0.483440660965608, 4.771649004345217, 1.7208980676435162, 0.4337330566653642, -0.9221188492469806, 0.8146078861683065, -0.04990526625898484, -1.0097375104985187, -2.7100999618463493, 1.9675974504745755, 0.9711901372115753, 0.35241350367940805, 1.4062642381285382, 0.0379025790799393, -2.4238565877539404, 1.0855901402458759, 1.3528100153070575, 2.35059232864023, 1.3709029873569052, 0.17589956872583742, 1.499792532883572, 2.7655160277363446, 2.0800348722185342, -16.077349242966456, 0.14466680107181762, 1.2802700228244481, -3.4717546220657676, 1.2761566175247698, 1.2254752785041527, 1.4554946525220889, -0.31850837750112015, 1.2819950900182977, 0.8355861596549375, 0.12377595159125496, 0.9251952843406421, 1.0443644576939204, -4.85831131837296, -4.023286461053084, 3.282047039089198, 0.6132525053505045, 0.20251433999936844, 1.0948003966666968, 2.18007669787312, 1.7943303489248699, -0.4270683299017781, 1.669717531209402, 0.706989821677775, 1.384751683825378, -0.47001233652765695, -1.0259334788653285, 0.2155162096491132, 0.2264134345733266, 0.37868420307037465, 1.4408764393791647, -3.905491723983655, 1.233826864184611, 0.24461020784119838, 0.7985864225270153, 1.2309052468778277, 0.8015045496697873, 0.3374904990572315, 0.8060623208710632, 1.2429985560840409, 0.8046089910209454, 1.2411335617226913, 0.07245522481692346, 0.9549757496661605, -6.182411828556346, 0.6279277677109709, 0.49581905678031457, 1.6939519318466651, -0.06613545254991207, 1.5204667559919565, 1.479114088598581, 1.6075266422752377, -7.249395315695504, 2.1442625523321284, -0.8618719990167646, 0.22467017197185687, -2.7845823422809866, -2.3857036713567212, 0.8252067950066402, 0.53428980112018, 0.32761507556258895, 1.3579414678060593, 1.3705369604034159, 0.7250552448337644, 1.1264741281496164, -0.2619336731402683, 0.2099346039765521, 1.1779708879954733, 0.6676676700677093, -1.4804381231346961, -0.2912805204835527, -0.47747829163319755, 1.1497400741172654, 0.7422620334243256, -0.48822259697939696, 0.7802009911211867, -1.1270702076894648, 0.5719760702162575, 0.5666045837773177, -2.4735110056769396, 0.3886996758699155, -0.3846162995670478, 1.2194011455371072, 0.9064592053786598, -1.1115684759803277, 1.144994536314216, -0.2778529438986078, -0.3795250141989045, 0.9935702334262035, 1.0690016662158128, 1.7100441137550317, 1.0855689364984056, -0.3134083612404631, -10.986440188517898, -3.6233107558389266, 1.304246158970906, 1.8251366930828694, 0.8450024440158098, -8.159554546910366, 0.5056442666621253, 1.259543447532588, -2.6304351750751382, -0.013029411588009493, 0.1766848592468054, -0.07568736706566045, 0.6931013385533723, 1.0313043105419017, -1.9442166222603048, 1.6871562339127586, 1.4000686086527845, 0.54893051362588, 1.3731779865835902, 0.5548592120124818, 0.9325171875423848, 0.5302412195918459, 0.44507809106555307, -0.5810088262342673, -2.0641764151808575, 0.40739314758382805, 1.3441594311851983, 0.19177442008026555, 0.5413233571194657, 0.21631100140145243, 1.2151209788137458, -1.960371966629581, -16.24281793093341, 0.5546630093000592, 0.15555836074916443, 0.06151421347598238, 1.099062410015813, -1.526446049509492, 0.4441019779207639, 0.7862177323204219, 0.18944710662490674, 0.9528122880564637, 1.3908569592585693, -1.0321723627979849, 1.3343176564598276, 0.28401516713320607, 0.7511886337102169, 0.5730526705203908, 0.7770624760736736, -0.6275046428979514, 0.4470002473070207, 1.6726864690128516, 0.45452171404666675, -0.6484110375279802, 0.10872614399817179, 1.3840568298756932, 1.0908003696035524, 1.5878095440832867, 1.3652921889406733, 1.5272743878507675, -2.2654832092385626, 1.0829335088550334, 1.4265451580317094, 0.3101052143640292, 2.036958327342933, 1.0200069126805127, 0.3190705642318094, 2.482235507945419, -1.258126499665072, 0.7859725996686212, 1.2949273225579854, 1.7322668420314606, 0.9522698720863939, -1.9529872178998289, 0.9586302880997121, -0.2878072301528115, -2.1997664389783234, 0.22379785740470495, 2.0103382476753473, 0.6208517383306041, 0.6908443036484959, 1.4669260815678926, 0.49249369775454077, 0.12800669433179707, 1.1788357720957863, 0.5778236424932545, 1.1040078532485655, 1.0218024631425073, 0.3643870235696258, 0.8505449264940436, 0.9857791777273587, 0.706711652465139, 0.7538854267721481, 1.1000980289360425, 0.585748694280813, 0.0679800055169055, -0.5601372700680334, -0.4891575527807438, -1.4420764589852035, 2.482235507945419, 2.1413964407567767, -16.092512792525955, 1.3027585703275553, 1.0780335426049465, 0.3006707916086394, 1.7106737595923984, 0.5352291687031272, 0.940277865113314, 1.082661754455897, 0.41207830093824704, 1.5658877618861664, 1.218883328966679, 0.6430729043970577, -2.959814889656357, -0.8717852259078617, 0.6344358835697318, 0.014962536608854021, 0.28500908710407646, 0.6662144027851408, 1.1632306735452855, -1.5904818197945967, 1.6067025446477075, -0.6586629760960753, -0.49144447192989504, 0.6959628782225462, -0.18345162888819191, 1.4423681056818276, 0.6764641124664043, 1.492268842378431, 0.954069264390299, 0.8476412229367609, 0.10662685800957103, 1.4606271578753638, 1.301452422934129, 1.1393940139956622, 1.6766945213034938, 1.5961928668347865, -0.7859848125852531, 1.4763843981053848, 1.0763825620517464, -1.0648727840067522, -2.9366228673155135, 0.016129111750160824, 0.46012077421856046, -3.1306151511001437, 1.5891626655894697, 0.9079843877137874, 0.7912314045625017, 1.8802028023441313, 1.5702888596857418, 1.6259543019300093, 1.349312557493963, 0.08046653614915786, -2.4780612020197306, 1.15001731685446, 1.1630591614404284, -1.0797761836357371, 0.025677538377123194, -0.5929285712411452, 1.7943695337008438, 0.3432473523485143, 0.5377740723128521, 0.7815534122311215, 1.3376161185755169, -1.085553153922177, 1.519480123114498, 0.49942600471953535, -0.12789336175910934, 0.33969198397898515, -2.2890335152109222, 1.3394490167455497, 0.17563350727708857, 0.9357987162320001, 1.2781076300412775, -0.5780760129946781, 0.8760861692240024, 0.48624217624365773, 0.5122706546375391, 1.4592883090771218, 0.8680751620031363, -6.629022296683924, -0.8141122626442886, -1.1158414566983317, 0.4053494684033103, 0.28916795958825015, -1.0520469293874035, 0.28735322260017787, -0.49131747448930174, 0.10171237790691831, -0.658158072769771, -1.2020465646658434, 1.5560478545891743, -0.8016096391823392, 1.4550617265583188, 0.9204035989986539, 1.280465956169447, -0.2626330754541558, 1.7055304848839334, 1.2445408302951422, -2.08776556053097, -0.22270893167294462, 1.640081845507313, 1.6288273985846633, 0.4987289309283225, -0.828663318681293, 1.5288810343755348, 0.7843588551664664, 0.3721589010105065, -7.251060459646875, -2.7995064447030935, 0.07685496746891712, 0.9106305746537634, 0.21039907276328254, 0.16489496459837158, 0.38586418275615286, 1.5646374681578437, 1.443163882523759, 1.3649785741393734, -0.7943977973866491, -0.8026102538178891, 0.7658537095670034, -0.36773151845337304, 1.3774879920210643, 1.2585994351024965, 1.2765187254445995, 0.8221559474499586, -0.551677441303905, 0.2517611664455919, 1.9186356845702865, 1.043466708322909, -0.8716641673125688, 0.3114754015025619, -0.55007485902141, 0.6939897340581521, 0.31505620189402195, 0.07063758442365656, 1.4927969901093063, 0.8805039951390531, -1.8399182251931714, 0.8296143758619555, -0.1943406127077247, 0.19852633291722807, 1.6278419697349116, -0.3338960827527654, -0.7557997745943568, 2.3980818585320716, 0.8361998598700452, 1.1815048807053021, 1.2166694451911315, 0.2777893960077371, 1.0500221412258977, -0.021108025450666035, 0.6733796000664422, 0.6533142275610427, 0.2841594477432987, -19.639338935351297, 1.0927037948645577, -0.46294585229581264, 1.6089184077412086, 0.6474664325288948, 0.31038257076720666, 0.886756883855881, -1.8661483827640535, -0.8068718379516399, 0.7220178238024099, 1.2468868596361342, 1.0058705574216318, -1.9892230003221743, 0.4832970640019918, 0.1699896727321211, -0.05661573452236035, 0.08900792568180071, -0.8016096391823392, 0.9150860121464841, 1.6338428146789299, 0.9453445171830347, 1.871570190929741, -0.36009914800874787, 0.5024708037872718, 0.7196455052204926, 1.9428129791606081, -0.4309842103888717, -1.0036613884885774, 1.3514917763944392, 0.4550771959695402, -0.34269029492839387, -0.12288918080621239, 1.504882901745191, 0.9515153689584034, -1.5916507819000278, 1.1288412337010814, 0.3828764026660347, -0.9116668845152459, 0.9700237850627872, 1.2568800131868483, 1.1077336073172057, 13.523620149443797, 0.7591018888008535, 4.703755279991948, 1.8547872725146048, 1.818440438677022, 2.3354374580806185, 0.2871122043132722, 1.7839161659744764, 1.3507904682356786, -0.10814791260010523, 1.127262379086728, 0.5177972876700966, 0.40898628633629774, -4.702935981928881, -3.0004767819833926, 1.8644747744197379, 2.2566537138446803, 0.07432631358946348, 0.12444888439438581, 2.115899045604744, 1.104813718760394, 1.5891530665027374, 2.482235507945419, 0.4542502909619346, 1.1705511115199478, -2.1293411399988855, 1.208522439487989, -1.0013198604527693, 0.5625858654949297, 0.21914208118060804, 1.1805313379701108, -4.468243045961274, -1.0098884101612648, -0.7313212191698998, 0.32209523386442274, 0.28384037100260395, -0.8304163388109825, 0.3213776291860895, 0.42305942650591055, 0.4509427290674578, -0.22195196437348333, 0.7210248174456632, -0.6558709338398008, 0.9270315472189835, 1.0503049091641474, 1.0642597493948374, -6.948450227280105, 0.6704362277286398, -0.22613779942926895, 1.350604673171458, -3.93690407052664, 1.2023753741534975, 0.9324194019702965, 0.8281163567108464, -0.1749576174548985, 1.621092262445705, -0.5344698059618904, -2.0674426483818604, 1.49207458734284, 0.5162048730315082, 1.2290793310845816, 0.14861284084146437, 0.03415313316603005, -0.6478365366126176, 2.0302719435180627, 0.6536481778915775, -0.28520112376552, 0.19167711602356932, 0.31233730382407576, 1.1152178961004324, 1.4857169886826675, 0.7894229516132512, 0.3176828771530501, 0.7009550611720984, -3.427284159762955, -0.21186993373368848, 0.2816239984729531, 1.7503048273912616, 1.1097698178719912, 0.9670894876567334, 0.12107995779826877, 1.5291792523577914, -0.13772873503798091, 0.8886319543499416, -0.4227627111143748, 1.4234615887127189, 0.042704098172376886, -0.16054384134659924, 0.33114583135095477, 1.2474303170848657, 0.22842543274370536, 0.9513071662041991, -8.556113559537716, -0.5405588001647138, 0.4885456738024043, 0.022734886047108493, 1.3162933343522165, 2.0768672355370805, -1.1566683234679296, -0.39080918978135315, 0.0716494499375753, 1.5471400317607924, 1.180833867211456, 0.43818825635444647, 0.5757806833473467, 0.9046602108638594, 1.0706896650838618, 0.6979334314252884, 0.5354051157647858, -0.26303774991082474, -0.7091492053988777, -0.6134589553820976, 1.7381110491046223, -0.003816389953906957, 2.100150073918959, 0.35322467721424544, -0.13162864457154713, 0.7140191758148288, 0.1010685808001913, 0.7045050274823402, 1.0056116002383089, -0.01071616279901258, -0.6702491834146369, 0.7238931165923241, 0.8220973760883946, 0.8848468131566789, -0.7688229563551133, 0.6506948467498073, 1.3513322244654982, -1.3876001312358264, 1.3924398769305317, 0.21369933914085407, 1.9061870366721845, 1.3002082996365447, 1.0825694635349488, 0.9965603683384828, 0.6960538800701329, -3.270118330659269, -0.36442515246484897, -0.6649382261760117, 1.0922092498597722, 0.09154540686568072, 0.3843303612126492, 0.8190680056674671, 0.30359343626671187, 1.4777370771334108, 1.2340346920376477, 1.389054632995709, 0.746121080554452, 0.6623928559563323, 1.2817797096763608, 0.49645716652054817, 0.4627941811375632, 0.09168963680530204, 1.5852319571661453, 1.7963750830369059, 0.2077521767591812, -1.3576414784300501, 0.8089033804618465, -0.5270001226399857, 1.393833450659508, 1.0828033616907806, 1.4748428479463025, 1.3986505100037985, 1.2357448823513206, 0.5023195549843712, 1.0455960550624128, 0.6144525862818084, -0.1174652649187852, 2.663274068861068, 1.1453841848766635, 2.176027044017307, 1.6100850421934876, 1.2701682847105362, 1.9417269437624185, 0.0636030342651702, 0.331422643808803, -1.2134803325526506, -0.5490294116485401, -0.27752639767591336, 0.47727443133266756, 1.1337735757409644, 4.550967573034712, -4.221389648706225, -0.08066571496842465, 1.4106341551101207, -1.529828968468196, 0.7084848399952033, -1.6866611081361125, -1.129805786906315, 1.2277189424971515, -2.8321403942562924, -0.7806642691604869, 0.9041463570102509, 0.18020983149762215, 1.3149065359355179, 0.7836604618337842, 0.5523997799119279, 0.7866396209052192, 0.7749381455280747, 1.3847514074401392, 0.7148977742510659, -0.540541668417471, 0.8199302868510397, 1.0799834843941283, 0.5581890779755038, 1.3960566158379955, 1.3418794593601613, 0.815412797071361, 0.2009021817181311, 1.0520370045326857, 0.6444415563101028, 0.9346186551802697, -5.545591318882288, 1.0194696303738506, 0.7943993438832645, -0.3531359593978637, -0.47271460141487454, -1.6201560857932762, 0.7777330644412135, 1.4290733227953913, 0.0654185956584605, 1.1894810461140208, -0.35663813417899104, 0.7487863949596564, -0.78295276643043, 0.5314316114919403, 2.301217538526182, 0.29181394574008424, 1.1618049567176478, -0.14528950834889262, 0.2678082540046044, -0.9718064973066473, 1.827767827525423, 1.5152625163065763, -2.392532807615233, -16.082770844114687, 0.29274334777942235, 0.5575117660840008, 1.2115136998087321, 0.39407087976036215, -0.06459996248742186, -2.4190892421363817, 0.9775864226871132, 0.6254266536024485, -0.09395644444458043, 1.2143780550339565, 0.6650838312815639, 0.45822437738933675, 0.6549490794478258, 0.5458520876206076, 0.4767971217808339, 0.7653560828692011, 1.6008238963888912, 0.702409683282292, 0.5054414655386106, -2.0572878622016395, -0.8016096391823392, -0.07508041226686432, 0.5971771836021814, 0.8331398002913367, 1.283768367396497, -0.18300780471401992, 0.9932157995579246, 0.4852675981763308, 0.3383480420170969, 1.4946553999953345, 0.803073859021445, 0.9566216727706354, 0.7665844570685898, 1.621319347824242, 1.1770096114069657, -0.36459142717841153, -1.0741688231021873, -3.6459719361623955, 1.4129659554481833, -0.778412320998349, 0.703373359612835, 1.7399202555619289, 1.5572827557865752, 1.547459760187644, 0.8975298610227537, 1.605604300173464, 1.4056494449104835, 1.1549876272161899, 1.1347646397810485, 2.0288994574253456, 0.9904702425593057, -0.14320569844871725, 1.4596573598261942, -0.9784009602891085, -0.039481808772679206, 2.239718850666078, 1.4594972043493595, -1.2683164338795156, -1.3096496206389356, 0.2109332852643873, 1.1963686094083539, -0.03140795176791114, 1.399623013913783, -2.616904803269713, -12.99971815365777, 1.9541800108630787, 0.8225310816146088, 0.8513341378863031, 0.40561803087852627, 0.7237449129623587, 0.0721101600496252, -4.484537443360082, 0.894139730330615, 0.2526275495849943, -0.34452540363607886, 0.6639206791274361, 0.14504456893946363, -0.18597837021406374, 1.2097648605743554, -7.735592929149024, 0.6907921923001737, 0.5828966670416278, 0.629905486999144, 1.5298215603890037, 1.4179243966527801, 1.250200598002527, 1.3775738105029005, 4.220370791896863, -1.0985332032017876, -0.3896996907803608, -0.9250862988848284, 1.0172814674456643, -0.1906545626959145, 1.6110349047534038, -0.1774683796690534, 0.9526865574421469, 1.4864088223836804, 1.1083091572233106, 0.5511616101766513, -3.4326891349654214, 0.281229970965312, 0.1270023220215921, 1.401937105454947, 1.116308095631499, -1.0659939700874022, -4.59941842698869, 0.7046970390055455, 1.5285947066239, 0.21438737604426278, 0.8369998898398052, 2.4511591636042604, 1.4488330210408529, 1.9272802484313296, 0.5823119906855665, 0.14045273863369617, 0.31613589411149057, 1.0003866303500344, -0.15150081711454047, 2.1413964407567767, 1.242751307705718, 0.5981574523146467, 2.134121120769959, -1.1153899582590616, -0.3257624578502515, -1.6418825934900034, -0.2695187308347853, 1.4468324704631303, -0.29432983809291524, -0.36016985741870877, -2.0112352989403033, -0.2233118283829241, 0.6407450350637809, 2.1234666522129144, -6.78185950100538, 1.6375439986405262, 0.7163038950653416, 0.6506251737706777, -0.6222746042809534, -0.433270243322481, 0.8428859099385839, -0.5991432652712593, -4.276142674464295, 0.47047885282862, 1.1459934908419886, 1.1896799055253446, 1.7311384969296035, 2.507058637637934, 1.246620084218448, 2.1972456595092873, -1.4216459062334506, -0.26613292095416596, -0.26809092642854004, 0.5035211840898685, 0.9418348773349589, 0.8748890647930221, 1.2677823202551346, -1.5496294794823757, -0.9011188932713182, 1.5851390146191129, 0.8141537115644291, -2.3999092854115296, -2.170544858771372, 0.13583763077437597, 0.5668981066868365, -0.6067737656778082, -0.8678439489747515, 0.45840131139963514, 0.3474983712250666, 0.14976451005051178, -0.271906309765628, 0.8506235327578349, 1.517419306975359, 0.5608920178399635, 1.1277282722156727, 1.1750668837436737, -5.0188939491738775, 1.1484646024981733, -7.560133728771692, 0.4770735283529489, -1.0535630430131613, 0.8686632546747891, 1.4314887933187885, -0.7486506678854865, -0.44876769423710583, -1.1102771631183903, -0.1426836051325994, 1.358699907725686, 1.4482213117601912, 0.879282680796264, 1.5873115213305127, 1.7862275163398862, 0.6143088908839619, 1.2191093821601773, 0.42771959371465307, -7.164377295597547, 1.1780642074064689, 1.472217687996161, -0.16892831895376878, 0.14311625246543289, 0.36617118904428214, 0.7556986532509037, 1.6311805408024151, -0.0012289256957206673, 0.550855294164182, 1.0427082445643687, 0.8092775808218597, -2.8680372715445297, 0.5865025896537323, 2.071837132329055, 1.595760711817668, 0.5405025192068527, -6.245477636112222, 0.6311307604775649, 0.9959251615574687, -0.3294142193238849, -0.24865858119152176, 0.46415293005735414, -1.1438707053514425, 0.8341795125774593, 1.2805651127367672, 1.1457614738529975, 1.086923635163247, 0.7481316417101674, 0.16972700243914893, -0.9003578280508872, 0.4879924205259484, 1.044014157784012, -0.05599292904529765, 0.9482433536139937, 0.04119206011874302, 0.6381835269271605, -1.5088931893458504, 0.8548567837614682, 0.8340870355415606, -2.2264652090780888, 0.5676713745270063, -1.7608598677046259, 0.8030768851928861, -0.6861599290174994, -0.05603756054542833, 0.1689507065287059, 1.216819872992663, 0.29589140030661204, -0.37780391003789593, 0.9780951725519705, 0.4962196103479546, 0.5546630093000592, -0.12996111563659232, 1.0046989886592945, 1.5573787282678715, 1.2779482687829853, 1.2622386084952415, 0.9353268845110608, -0.2428058975826269, 1.2140082281622833, 0.9929422416726924, 0.9829492787152782, 0.6484997181219507, -0.24737581772836062, 1.6565205579438367, 0.793589187202865, -0.35635753896026734, 0.0885821960284138, -0.45794344264830633, 0.19505870915815446, 1.4364896125868742, -1.7315277396433582, 0.24839816306945428, -1.9366289290856191, -0.43131946888143835, 0.24206904151608669, 0.9324329809896134, 0.9440293015159112, 0.8456538725620487, 1.2961301321909084, 0.9313028765326506, 0.5249449028464193, 0.4219600516270021, 0.6988290379151176, 0.9311424676799755, 1.3443380205562607, 0.3542088459160845, -0.40797624491610085, 0.5861919170817482, -0.3952709250070203, 0.2735961364617934, 0.8510206182438231, 0.7010887025616678, 0.612787121900352, 2.2398352436639164, -4.755015778131383, -0.5930376827543934, 0.5360295507320328, -1.037436400360155, 0.6611751580601649, 0.025779003262316275, -1.498442892677121, 0.8588426768225682, 0.9541955294117849, 0.9859873888362204, 1.102056218994863, -3.6114291493394797, 1.7046789853951307, -0.19493594525142263, 0.20895027770771468, 1.6083187935945973, 0.7785574752727445, 0.39752155220896496, 1.8024429275670324, 0.7440301713730151, 0.3983199385436724, -0.04444844748636964, 0.39060217352319604, 0.605730608796701, 0.15932885897579552, 0.6469643634365689, 0.5052483719178174, -0.1108712134323068, 1.5567485261728986, 1.6385085941601523, 0.3533790746096295, -0.6304282111742406, 1.263518598251837, 0.29552455213773376, 1.8286556389867947, -0.7803619763135093, 1.4602846268174092, -1.0378976039439989, 0.8005214250725715, -0.051454659092112304, -2.86066088789977, -1.7430559390573142, -6.720306018091578, 0.385084356301132, 0.4115939852177041, -3.79457565185049, -1.2389779587821095, 0.4504869172401966, 1.4103928507335455, 2.255758507785251, -1.7148473088246063, 2.1711590717016893, 0.22330925978346175, 0.5638100579438293, 1.5622406041180603, -0.13344854473859116, -0.14890725331845392, 1.1452074110659076, -0.7605343886195189, 1.5861458469733436, 1.5762994910515054, -1.1272594330125725, 1.081842684230858, 0.9833654478430497, 0.7099874224120525, 0.5551248278638327, 2.0743975830036003, 1.2859097308871394, 2.1554814754809515, 2.272685124676139, 1.068652922045973, 0.2096332076243154, 0.14721719306292766, 0.7805647724370829, -0.135698753492716, 0.8784040244897656, 1.1295136400437982, -3.1586228898457724, -1.3241233003097819, 0.23917884487448637, 0.4177088467308677, -0.2778612755477212, 1.103424211990246, 1.6932973286558497, 0.841524012534424, 1.365040161695303, 1.3014977240772383, 1.1956511564914862, 0.09924284479935302, 0.29369048180242197, -3.1794791424124127, -0.024452706618780046, 1.846706040046761, 1.2227559313951295, 0.7060288540935037, -1.4892652859013347, -1.0127526060141647, 0.16847901043724986, 0.5815460227473225, 0.9871623949138296, 0.8554069977216141, -1.197129824389136, -0.09090692589514707, -0.049275351177152565, 1.0160400965500502, -0.8770396389101803, -1.9818734174390011, 0.2626400073020373, -4.569702696663064, 0.5357772531088648, -3.1691593695015583, 1.7567683838401906, -1.0049431318902327, 0.5087019876233311, -3.8531183447932342, 0.8973944567937955, 1.121102103268694, -0.7258673936143628, 1.4739204358680882, 1.6177664169408903, 0.6873643031440707, 0.3425242970849192, 0.6784637618372148, 0.9588607276180183, 0.6750216268673447, 0.9662538596809264, 1.0731182742288865, 0.7614883274217722, 1.1313059442920428, 0.3741098744295914, -0.16238224904864976, 0.3767514114278424, 0.6424942596310849, -0.6337139626384863, 0.11862183684109354, 0.48773487858510833, 0.8285135677907133, 0.027189293867691228, 1.2196663953964482, -0.6652695817825179, 0.8289650113781712, 0.3898967780228404, 0.7205957668408669, 0.3084407337128746, 0.9565446124346341, 1.6545009737560314, -0.6521064396675286, -0.0649126713741078, 0.20033564662089023, -0.2982317969231234, 0.28133418659444354, 0.4008859650697537, 0.43814429595369075, -1.3147496368839573, 0.019895621335162206, 0.5174536684030929, -3.8583757635795317, -0.6559264911300823, -0.8960579350597812, -0.20309894526781438, 1.5983200390852876, 1.0068285158869208, 1.191145070775604, -0.23445880991129855, 0.054536365450335134, 0.11414232026471702, -0.14148116810996844, 0.768544182153151, 1.2133747810631186, 1.093923634585947, 0.5198203860421293, 1.516064305166535, 0.9497438022153426, 0.5552282688216063, -0.9520704906973713, 1.5150081962014879, -0.5565705714875933, -2.8700180899611825, 2.2273095881609724, 0.11435256140878483, -0.49070173220440705, 1.6867944797900143, 1.4093545114579176, 0.981694738033429, 1.8515422994761803, 0.4758345049468773, 0.5521711474884636, 2.0187065212089816, 0.501439833440846, 1.258693119553047, 0.9580506820327316, 0.8250152129267341, -0.7863273675267962, 0.6924132764894811, -3.265729032681409, 0.46343762585481996, 0.537062733156306, 0.9092447604939234, 0.8580288431840706, 0.5452655530656711, -1.8705141863073889, 1.4559703110507904, 0.4772013363433518, 1.4215077031097016, 1.6337558202428177, -1.3244218894405353, 0.11775012178912825, 0.5599363153395863, 0.27597637387152113, 0.02266687025487743, 1.5363923597428026, 1.4704174603208437, -12.834402314409585, 0.8631469992436829, 0.20300756372126125, 1.0870791362801413, 1.204022086460079, 1.8802028023441313, 1.3236118192164665, 0.2857129708423264, 0.3399850551892084, 0.9390358721641583, 0.939814303066278, 1.080390145247533, 1.3856168295139089, -2.8082570786944228, 0.7342010323519026, 1.8555284081313514, -0.8279637549928404, -0.39162607192103593, 1.5372959874083696, 0.8908440272727487, 0.8014782019447086, 1.3338004774279042, 1.157243798033353, -2.6894508456610606, -7.035533656183886, 0.15288805026527155, 0.5070344735013503, -4.762844396000366, 0.9869042686353526, -5.113442480331704, -0.6115747041626216, -1.2033926765161904, 0.41850449948855584, -0.2807360153623438, -5.75650289981623, 0.04223717769238758, -9.385296968976458, -0.03523729151966429, 0.7368663412985147, 1.9236239559863126, 1.456435645654242, 1.1890582975855326, 3.7411667536536477, -2.6863770127401096, 0.7763232993757427, 1.02242852852967, 0.35934412150155437, 0.6454009792904354, -0.36086974944633277, -2.0541865577557394, -0.9332684755350006, 2.1637503590143807, 1.0384197931749632, 0.27221461444984674, 1.2523880623840884, 1.3799712798573942, 1.2752319231401243, 1.3045049365797967, 0.12865254329045797, -1.3180423507164367, 2.0062049971102205, 1.6965476310387935, 1.4865603754158454, 1.388598249840515, -0.4001584203424583, -0.7324059472314098, -0.6471798173167065, -0.12866562869737827, -0.44599114699184095, 1.411541693983707, 1.1347981532313738, 0.5610781321633485, -0.4185674449799842, 0.25954680230466154, -1.2460538703655355, -0.05973422052491709, 0.42163905884489566, -0.913596184887196, -0.05187608726522877, -0.44545012101689174, -1.0078503451620202, 0.7835470857182724, 0.957934811192153, 0.17909491598603566, -0.6673764801599665, -5.265574135408964, 1.6368595293046144, 0.3035383518568786, 0.08051358967884856, 1.1894407455533458, 0.9586339567928629, 0.44182390128863824, 0.7012007106373729, 0.28998938717782197, -0.5390949384840902, -0.2922139651451817, 0.7393782177657132, 1.609316390003604, 0.14612047141043505, 0.5639058202119988, 0.524308159692294, 0.21810578667831218, 1.7377284023585573, -2.6775363217695225, -0.12105403719180344, 0.6844957957753788, -0.07297012488569371, 1.2693787850845741, 0.5792941371572695, 0.47922391262903796, -0.14381058925485715, -3.6226102604240835, -1.492434011514926, 0.2524640973002649, -0.5346724117040739, 0.9700815780791382, -0.1060392594651822, -0.24317442668132752, 0.9751039580972786, 1.2988253823348435, 0.24402546756739737, -0.929550343321497, 0.1500164975997845, -0.052175309021187694, -1.2828954751914885, 0.6989043127106477, 1.119662193498015, 0.705338683857909, 0.310752617229316, 1.9221873661101099, -0.6227253685219816, 1.1248368324437197, 1.1667381824777618, 0.9530525773508539, 1.321475815135768, 0.8673619535594626, 1.046855516210468, 1.374517745528771, 0.04002416298259545, -0.23055855596446392, 0.3879043636755046, 0.08290724173555239, 0.05374018101526118, -0.18372559683359177, -0.872892902792984, 1.3750929537306928, 0.8607318764968647, 1.0981782785274783, 0.08154483836170132, 1.7407327897094145, 0.23913076816610632, 1.858086883220475, 1.1150363831797097, 0.6047791940349673, 1.7349923325404615, 0.8587070975741865, 0.15551701946115012, 1.1685923509539584, 0.25235080288455686, 0.2597160036400623, 1.1176336461076481, 1.7518335607527051, 0.8039160206829483, 0.7096066888350786, -0.07421608757387313, 1.2168761278677824, 1.021870169774284, 0.08972710497363093, -0.3256823857328829, -0.9295106713428517, 0.8046395502932838, 1.5255052924745385, -0.535725115505598, -0.9829203589371555, 2.1413964407567767, 1.5833164046964063, -2.817540396044438, -4.163860248284706, 1.4886040253513821, 0.7631766732857208, -0.5062898989191057, 0.2248819005655796, 0.5546630093000592, 0.04784427665142113, -2.118313552908842, -1.0873044561215612, 1.4841838585738891, 0.532769258439247, -4.291603676835877, 0.5190930521049538, -0.19136912921936935, 0.8495242293947936, 0.23131266378365323, 1.03662047444746, 0.5299558938001446, -0.8754056805226275, 0.7294575555276157, 0.10896638272008587, 0.7953107517175174, 0.0051099538100292385, -2.4302044384996733, 0.951222591633514, -3.2169146834445854, 0.7797184330806277, -2.136289549917831, -0.641432194017676, 0.6655451952226582, -2.8835972922654554, -0.9512445351792004, 0.5167759059935091, -3.448595339109971, 1.4174428102879872, 1.1282093749950057, -3.9171301475649924, 0.95667496233173, 1.0281341387394707, 1.7697901867219883, 0.8646163280164515, 1.063908077834993, 0.1467905481315914, 1.5775878206033738, 1.2107878457659682, -0.19356536167857355, 1.2282287966054621, 0.6763601244810189, -0.15257911081466827, -2.573844706180532, 0.1485980377497578, 1.552711581945735, 0.26051033125040546, 0.8488087363464056, 1.2173242466092216, 1.9359464379637414, 1.6731180451445895, 0.8523504374761782, 1.8638828098292375, 1.3407737633508863, 0.7599282875492531, 0.08715532003215717, -1.838206848305844, 0.7264431360403498, 0.394640723187607, 1.5158790127828599, 0.7805238573065607, 1.2364585275894469, 0.4673380007449526, 0.6061737387394234, 0.3607155505817623, -0.308009211280345, 0.6419981272000256, 1.5945197706448277, -0.7721077846362021, 0.9291337525143203, 0.521256105476815, 1.3201513126632511, 0.9133985636855609, -0.4845999764344091, 0.6662608795696722, 1.2357664482957693, -0.3221570475961556, 2.466399108159459, 0.9503936508136359, 0.9546735315974855, 0.5472924265438244, 0.1448745846602451, 0.06632410925524443, 0.7251538274600021, 0.6593278261340623, 0.948341510961446, 1.1589767516948575, 1.172636179243001, 0.9275255734785139, 1.1780168672526563, 0.8813155036952357, 1.4067243703313879, 0.42501784509968066, -1.3920813129607243, -0.02213187239245376, -0.9794588577671829, -1.0965217607513231, 0.6021080835419959, -0.8919377409550072, 1.1449523139373743, -0.4604591061774287, -0.14164091515848734, 1.474875259570357, -0.6208878990659313, 0.5334835840414653, 0.08744061545298845, 0.7687893941032895, 0.6957727213214598, -2.8220155808125513, 1.2824222977174315, 1.2522368437462743, 1.7059537442805421, 1.0305480299169236, 0.7696365483562266, -4.41053601833171, 0.9938487152597583, -4.268968128220799, -1.6793676389258216, 1.3287038008763004, 1.1874669943659157, 0.1426970822069763, 1.010399285163724, 0.8633723288515989, 0.44735838786979165, 0.7149113267663558, -0.9529841068305694, 0.9754267273079792, 0.7394317706062378, 0.25985235667161, 1.6680433795983858, 1.57385769034087, 1.5841800915329842, 0.23524431335656207, -0.6312344183755129, 0.37710319341876253, 1.1418417487691368, 1.1094602479737463, 0.6610137919387491, 0.12912123485621116, 0.20179488886145325, 0.3951456791948772, 0.08870807098114278, 0.1183400416215443, 0.8231278452862503, 0.22129963068747005, -1.3651636211135096, 1.6327662136160237, -1.9210188396188623, 0.41774524651584555, 0.2565011626750392, -0.3152354830943538, 1.3903639483327452, -0.14559642003763432, -0.7414966123043, -0.27279467477992814, 1.444873745265215, -1.884612643160679, 0.5571586991322358, 0.8235720151599598, 1.1904380638635017, 1.666839817520999, 0.04446588201678078, 1.1301825780557948, 0.7337090488037226, 0.6766382746681293, -0.05656437879844009, 1.1316166432880168, -3.5461818457716903, 0.7355255794412414, -0.047823852439355524, 1.6877787208629305, -0.05110624221186619, 0.1422861574798168, 0.829768422594743, -1.4660637057881662, 0.6720475528033555, -1.2233612873900745, 0.06979777528380857, -0.013145012697163255, 0.9695808047457529, 0.8340488322339182, 0.6363964312455396, 0.2429708478071432, -0.32352515041360186, -1.030340934683913, -1.4449691380853165, 1.061709492581296, 1.3751780956249047, 0.7599282875492531, 1.6617445929320358, 1.3460369978276114, 0.3012724270380316, 1.5718394609083577, 1.7513503586546515, -0.8359617684138206, -2.0318420130380286, 1.761490639511597, -0.6659371220765823, 0.8634917468036913, 2.039858696512264, -0.13433514965035318, 0.11591750005006424, 0.7244366479204669, 0.23653957704640013, -0.19022681503525155, -0.17989763487152569, 0.9470010127977097, -0.09926688292768712, 0.31206436315006425, -2.992840574905342, -0.2449652487919012, 0.37225141592657424, 1.638276948367571, -0.60634799183194, 0.8954420258577225, -0.5025643703380857, -0.6435066518179826, 0.7157372131579885, -0.7009120727370826, 1.0443487110653815, 1.526645720526016, 0.20720473301057896, 0.4592997631194533, 12.075633575492434, 1.6121879654187323, 0.3494204085381801, 2.275186323537906, 1.5339002991284258, 0.5731709660302599, 1.157233963011786, 0.1628702917672555, 2.03241374915377, 0.15806875469614942, 0.41832112892060425, -0.03684426636644919, 1.8816901510179178, 1.6653225104716873, 0.5300329913809113, 0.3624738872698934, 0.7449310759706577, 0.35021980890786797, -3.939455792541667, -6.8153738395044385, 0.1643497119563264, -0.2001066505552537, 0.7743585333209814, 1.3270481530294906, 0.5232805998156532, -1.1990190329134864, 0.39353285209320554, -2.3578742432232302, -0.012978163440769713, -0.8016096391823392, -11.880948995008932, 0.8404699305199835, 1.4505609307462735, 0.7560166760456589, 1.4006184352860565, 0.24568221241702634, 1.292022884140067, 0.10471184376115059, 1.2273521840920985, 0.6912147685506015, 0.24718871150572655, 1.0461594482754482, 1.3013956100378674, 0.2189621270233757, 1.043813224426396, -1.4080871261052226, -0.7217259842931913, 0.908268720651015, 1.3028319191708988, 0.15393764707767618, 0.26430244853545515, 0.6563110410536799, 0.7363653011657479, 0.022626827702159985, -0.4831724858364266, 0.8371035059906263, 0.1530005379677428, -0.27737102297824523, 0.4415743312356849, 1.800558294891469, 0.40797821523791955, 2.0685170547627942, 0.10038202385077369, 1.6478358150978514, 1.1580453979030303, -17.390274400374683, -0.8888258028308319, 0.08375221180366843, 0.8295989705311865, 0.9298077562686847, 1.3534260696631937, 1.3971592123210894, 6.293313951206791, 0.27654283914544475, -0.8264126292123174, 0.873852547258826, 2.4476417157899184, 6.979380393247012, 0.4996025935243872, -0.29796814445787634, 1.1564439423057782, 1.0508264375509546, 0.9351508837977252, 1.206514754500016, 0.1058036957361079, 0.43630573371323056, 1.4794273973513283, 1.534322917409025, 1.1176336461076481, 0.11666755987151753, 0.9079177242648976, 0.7995859754932697, 0.5639868543192194, 1.2773620668160006, 0.607831178992574, 1.2404966292378823, 0.9706327178981585, -0.5313319597889752, -3.762263447485083, 0.2167404686496149, 0.3193434429678591, -0.2909850630933202, -5.53181579515318, 0.8469008853769661, 0.9190655362784879, 0.8430727987958441, -2.408246968357969, 1.6014241969162504, 0.8499891740591129, 1.9241717423376, 1.653945422706977, -0.45047265449556395, 1.4612515258269558, -1.0966393513270583, 1.9348724244399773, -0.20785653320165284, 0.924339577757649, -0.7113179049028145, -0.674255587494233, 3.4321733277908084, 0.09937888501940659, 0.30838205879683356, 0.08673358695653646, 0.7372076731899345, 0.5490428667977784, 0.5770558391706619, 1.5560478545891743, 0.6309065904491027, 0.24090486943305509, 0.9464633879479492, 0.5410990108016012, 1.2723641902511076, -1.7681799199058683, 2.464403166616136, 2.427813109724834, 0.9875181593442177, 1.356492108742731, -0.21053070973789548, -0.1015165681329964, -1.385568938432309, -0.44547518347403536, -6.104046693099702, 0.7281477788988838, -0.1621186402036863, -0.8858077391205909, -1.031222870458345, 0.5791322299825894, 0.4620117398016338, 1.1990923358461707, 0.4204922194261996, 0.5662043203745415, 0.3873644668643992, -0.18243115277419156, -4.416793365975003, 0.8235883654625481, 0.4829277537847584, 1.2871613242499382, 0.6882707308096906, 0.9441519160272605, 0.4679949040603695, 0.8592847202122904, -0.4027880049957614, 0.5172672616509019, 0.10375021861175801, -1.1850526800126353, 0.9727164142714947, 0.6931633573656429, 0.8790985527820389, 1.3920636793195338, 0.22545520428673718, -0.3648068073794053, 0.8835135308382642, -2.1135429965116517, 1.5902307877312851, 0.43688136986890425, 1.0260720677816204, 0.1688824850568, 1.4084616698694794, 0.5280350155543493, 0.48410667530400453, 0.21524028806734755, 1.2541323162842135, -1.944320095218689, 0.6527896138583815, 0.4943217944572238, -0.17474959800547987, -0.37436026770829856, 0.0675709054230936, -0.15804290061694073, 1.2243121515929574, 1.092944712534379, -0.07657215545398302, 1.5702045665371458, 1.4048875053241865, 1.6833595842718674, 0.30652967123699015, -0.3806180697661129, 1.9937939692894984, 1.4641531081740673, 0.48377793900877475, 1.6798498374777202, 2.4850422897227755, 1.0446680828097654, 0.8631727728278648, 0.9237883471286886, -0.18159212754705686, 0.3063413145832262, 0.9669508555191746, 0.9220166774734333, 1.7744489428411607, 24.74765952506842, 0.6481926614646673, 0.18928096140422887, 0.42005109740550206, 1.781451320341697, 1.6052139919415718, 1.5302077402031877, 1.6471848868337962, 1.7075772758057803, 1.1172292790369325, 0.5129464061573081, 1.2019836620383004, 2.0781120607842207, -1.8826099488830152, 1.6947405476091457, 0.09358680823463886, 0.40663138991269376, 0.8540848004098948, 0.09172532678420828, 1.3057793056613676, 0.49271144264249267, 0.8227933479219963, -0.6197212955850785, 1.9668251485968697, 1.344962897487811, -14.053129773664411, 1.871570190929741, 0.32784333922551523, -5.341259839667669, -0.012892950016478583, 0.5040227214115951, -0.6976477380492428, 0.9865888116826791, 0.34468029744926854, -2.5884692500583784, -0.9617232946897202, -0.2656307762564747, -3.5969239140497478, 1.5080161365882463, -0.2246019832951384, -0.033540478719126604, 1.2478742954976592, 0.48469437908033375, 1.1768730934147804, 0.49853726260821174, -0.16553832176695524, 0.9149877053613398, -0.2726805733064702, 0.144251642049047, -0.5619752835967958, 0.7149508586616867, -0.16506217149604693, 0.5914045240381681, 1.390781541123906, 0.10666178992628828, 0.26699439131477953, 0.11461021128304333, 0.4244032236076881, 1.7099208298072204, 1.1347050509341832, 1.1037265668523832, -1.6689298645199535, 0.7415483309571826, -0.5190229071610429, -2.1590999046026793, 0.337442491352306, 0.8614654033032177, 0.3895842679449223, 1.1622198658154632, -0.9682749132658962, -2.6446965694276097, 0.604418142728303, 0.6441656581811886, 0.8497008815660996, -1.8041658448087154, 1.4035031314847435, 1.2089668710968444, 1.4212892843520846, -1.456366335910774, 1.0429023108633437, -2.5885531893520533, 0.8042634106299069, 0.9756271158488691, -1.6732013307744473, 1.1552474870764962, 1.088830071727293, 1.9923745772644301, 0.4592191384511759, -0.23886342509303296, 1.1100169182557105, 1.6297431018033843, 1.4209950917117682, 1.902986998755226, 1.2660638637248904, 1.4640326191494208, 2.363578654377944, 1.916128223897262, 0.17358061700718538, 2.983102874703047, -5.610275480745319, -0.09191335002432574, -1.2542377486399752, 0.6555936753910659, 1.050741868218998, 0.07627724513565412, 0.5076717007504243, 1.117007135036358, 0.8010073080221148, 1.4428358389165827, 0.28799895483664206, 0.3227339645582016, 1.5795980974656925, 1.728742325042309, 4.248423452437747, 0.2825767546956507, -0.11859256227877998, 1.1925751702834957, 1.4542828147803804, 1.146494621190295, 0.6080410678447389, 0.07447856772556358, 0.4174507022569671, 1.179469774906157, -0.27762735479981615, 0.1185703599537544, -0.2763869663903017, 0.5951024244781326, -0.2741517359301149, 0.6443134029029295, -3.0370744421384246, -0.13446276258243747, 1.8315325122209873, 1.3986094858116327, 1.5167654706514335, 1.2447103237963826, 1.1902870236582772, 1.3240902148492137, 1.353296451272526, -0.6455745559377769, -0.2512319293730787, 1.692066513856211, 0.8639386020400864, 1.6128441607369834, 0.9345405179157243, 0.4035661008210111, 1.9425181418027264, 1.0206472136452855, 0.9098121543309062, 0.083037898674438, -0.10917130572806313, 1.6834078730942303, 1.5683943120233377, 1.0752935002551176, 1.8499274962379137, -0.48848421275882226, -1.144258063818827, 0.42451083584738125, 0.6618995308324087, 0.9867216851224178, 0.9307235494821212, 9.574655185121582, -0.6139657674891268, 0.7728759960866786, 0.5094833173609021, 0.8817785669236252, -2.936131837051791, -1.4800747064398132, -0.5419979436489654, 1.3653717534946936, 0.7704091318697194, 0.5237990086769947, 0.7379198430426068, 0.40635377888174284, 0.5061445279682437, 0.7650618988355004, 0.06873588551368523, -0.37501678115015513, 0.19393735346867022, 0.7975033591363155, 0.9814885412947053, 0.9241360126569707, 0.7275076465168971, 0.828186547122668, 0.42851956586173456, 1.6045969103867554, 1.437516673164168, 0.7120928007275165, -0.7508592568047879, -0.8182061522981107, -1.642693435954473, 0.056411191162474034, -1.0078580210023282, 0.8927055462117582, 0.9956405618089924, -0.5231443828598189, -1.2459507917282007, 0.049254805736841184, -0.30426641420246536, -1.6162268638513668, -3.6088333299133852, 1.3837140399932117, -0.4307522145643632, -0.8949998955732369, 1.4742867371580697, -0.6533311252005535, 1.270828471312191, -2.971723619959605, 0.9111104903072789, 0.3716673502585883, 0.3051613094238148, -2.516329813557115, 1.1052091968650821, -0.3730635242605679, 1.079230691874243, 0.3758074823949287, -0.2277028888529888, 1.0346898708354173, -1.0372029547215045, 0.5597320911085008, 1.1244619822325705, 1.4532216667250295, 0.7121001322772538, 0.7068231269270046, 0.5009112223435228, -8.715502406797535, 1.031654580837997, -0.4886353717959925, -1.8201762697966415, 1.4726711681624463, -1.6418062045269022, -0.6917105264139776, 1.0625511490101527, 0.05873438392077143, 1.2928097940288372, 1.4022243176679605, 0.3097270855743964, 0.9645635324248064, 1.3855429207458962, 1.0846049548002739, 1.4377522957808353, 1.1007270358456143, 0.9423339128985917, 0.7583616604897437, 1.3602108900902528, 1.310327805938813, 1.514744272726235, -2.8287832056251627, 0.9421822899541397, -0.4206412681939856, 0.5841055397501864, -1.2153721570074985, 0.39677135707167976, 0.31217508824279183, 1.2622833105035214, 0.45072680125595643, 0.9896322819530369, -0.6270029485087111, 1.1023581982069672, -2.5507051144605524, -6.540687538476149, 0.7957459856152397, -1.701106329327193, 0.4849635977363741, 0.2886965116395191, -8.644883505178312, -1.8785055358789433, 0.48505470907603987, 0.32446253309264567, 0.39954106648096543, -0.061128096729630016, -0.45596451478098454, -0.09382176359676289, 1.2943779504044457, 0.6763996535662687, 0.5872832990400507, -0.17398571802267696, 0.2597753819472899, 0.5182206123419884, 0.25815997441696686, 0.7510591167052472, -12.15116245638403, -0.3864401400851324, 1.2666831961001421, 0.40789122986977766, 0.5646012775390972, 0.23970485276115744, -0.9582307562886291, -5.341889583652342, 0.5062350340023559, 0.9525100675662262, 1.0391873444743376, 1.001888217540541, 1.0011433806327652, 0.019592536088469975, -0.7220920987564154, 1.526119667393837, 0.26072931887246287, 1.358196926427129, -1.6726762667196158, -1.1678809686765592, 1.0753714784424933, -1.5372428278743995, 0.8657241491203306, -1.8320710035736394, 1.0356624901035774, 0.3896131246475706, 1.0605083898740078, -1.1638961413872602, -1.3198009535894897, 1.0462009161975458, 0.6204497600089913, -0.7510205476014866, 0.3174238391965174, 0.6925135440917701, 0.4633442006586207, -0.8616391456461546, -0.2975426521686879, -0.034368664646530295, -0.19431186494553213, 0.6259866381816743, -0.17445200177421996, 1.3613366918058571, 0.5580522677780286, 1.0377727876991532, 0.23539063882608585, 0.7507021487942146, 1.7047960718988104, 0.6970589295712455, 0.8884384987172055, 0.39959229333498403, 0.635475060350672, -0.7591667476723275, 0.45812349615415326, -3.198351847723439, 0.631078048052866, 1.712789366009788, -1.3273160849598662, -4.405957383865951, -0.6440584955241918, 0.8407299266106733, -0.5588928425767341, 0.7264135503444621, 1.6589982400728007, -1.248941060212712, 0.9941418960722782, 1.2688876575145676, 0.9552553781078346, 0.8910523175483147, 0.6161025120640099, 1.464738132174695, 0.8166270159703661, -0.8016096391823392, 0.5207174223933388, 0.9979905965554114, -0.7191749733536253, 0.35811795795375556, 0.680159699051823, 0.28810433593901574, 0.9979905965554114, 0.016512937160724522, 1.3895795799050328, 0.8083114922034448, 0.8498946345183659, -4.550047292652385, -0.19740326046689752, 0.14219274867383078, -2.33968106726246, 0.13912752098326417, 1.5403260814882793, -0.16972536948639902, 0.3658082729478631, 1.9187794459393326, 1.1211942382104454, 0.8137911145474706, -0.2912455448329627, 0.9929916096203164, 0.08739490640590256, 1.2688884295100367, 1.398025581900969, 1.4970427465786575, 0.6465826599281861, 0.0012809036578496635, -0.4112716232823377, 0.726912451302392, 5.260713667498147, 0.9081921914554326, 1.6663318629967172, 2.7835659548402143, 3.795865522456368, 0.6278677189663477, 0.6352655064009864, 0.7359107360425956, 0.6885488719724232, 1.575474810462176, 1.9024815919757885, 1.0334926497723322, 0.2108031955643628, 0.5088079105562037, 1.2627702843838622, 1.4356223435810445, 1.6238099570487212, 0.23042860064829854, 0.6523820505976129, 0.3743662962007273, 1.002348361202397, 1.6344906270411579, -1.0225667162965786, 1.9795958220805228, 0.5798707044005615, -1.2240753852031572, -0.10128056562000096, 0.10067600597518103, 0.13765598218099004, -0.7401306599036185, 2.2359767669311164, 0.6341604461415319, 0.3707327376610463, 1.0439612344292046, 0.7393280341636538, 0.6159340075599967, 1.084854416256376, 0.9871542657699606, 0.9944829633510978, 0.21858382522813177, 1.1176336461076481, 0.7764476726715314, 1.1231977275466767, 2.204220489932329, -3.1274128063303497, 2.7642456897022814, 1.1506795385423039, 1.9359464379637414, -0.18414587823310982, 1.0445198697701819, 1.1943601262975896, 1.2199506267083124, 0.5464233854416776, 0.521384426671839, 0.9085659909714168, -0.07676899743720791, 0.7019870462766382, 2.388869005498489, 0.5527343127956421, 0.8063142220837319, -0.3947425184133393, 0.3706793269074605, 0.6051901446736502, 0.5271194333492148, 1.204754641265512, 0.9098347904880668, 0.5096191383786096, 0.8484759276753321, 0.8311608858030549, -0.8227019809692668, 0.3322267372988347, 1.1793092054595948, 2.2096579656272692, -0.1345584528854755, 0.11011732295382007, 0.7599282875492531, 2.309497814874752, 1.017818003391893, 1.5249484021367885, 1.6274271761504824, -1.3338157098959578, 1.293571488777525, 0.5772482256115535, 1.0477950896811246, 0.6845089695879012, 0.9190588201588284, 1.053464386142379, 1.194887724781078, 1.516273672649831, -1.4297489259898757, -0.28069842963605096, -0.5898258465673755, -2.671327321366813, 1.0642335489618606, 0.8344970011274627, -0.268392067800996, -0.7038940422091556, -0.1421412681145249, 1.77782794290881, 1.1318748116680952, -0.5880163378578289, 0.6814232801837032, -0.7917288085441044, 0.8995209919269359, -1.0005602767822908, 0.6867481880614844, 0.8388670963973651, 0.6681238419297506, 0.6120305434570935, 0.33744211504388644, 0.8999407037752073, 0.3940403637778378, 0.7005074598359875, 1.2704129052975073, -1.6587616868677832, -3.480008475944015, 0.30780184665305493, 0.8219808895852649, -0.31088040800547934, -4.210144467674038, 0.6069865591682181, 0.24656726864201212, 0.7931437230283298, -4.569418804467242, 0.3483998651555845, 0.32438104747220664, 0.8393768643824783, 0.32564330181977885, 0.5558514569024997, 1.1455971460321435, 1.790709832710018, 1.6121475981650009, 0.8274136523933442, 0.4722155671215382, 0.8716500063968259, 0.2665784548287824, 0.5843976228286228, 0.9960145993091035, 0.30570901005228135, 0.504473005613077, 1.306855907808279, 0.9982419277682498, -1.4603327759900229, 0.6081742368046966, -0.8075532316170616, -0.3097254232762855, 0.07487839322399151, 0.8368538534488016, 0.6200153188392786, 0.2745758543140455, -0.012772147032053122, 1.6122221549098115, 0.5360587522029943, -0.8182780897983152, 0.4516977770447199, 1.1198960725621965, -0.28468561333370845, -3.1072090992623798, 0.1671435036289024, 0.33074957956031165, 1.5519256130104868, 0.055405737864258314, 0.245085438934682, -0.5033641375419252, 0.179863954324438, 1.59418604098825, 2.3959550720746594, 1.9289313557598275, 1.052299210963539, 1.2130751276245981, 1.7629327097832994, 0.22376963627021512, 1.4111271685568794, 1.1807515945826081, 1.088662090434941, 0.47986931495635865, 8.657623536529018, -0.008765125883230751, 1.075789459754184, 0.9871504295150556, 1.6745668459571839, -0.44611922581936936, 0.48550767725005417, 1.4755780217068175, 1.5124676873656293, 2.5858369062299973, 1.7785356812923943, 1.114238038030237, 2.566877029465069, -2.1604545057477784, 2.1012083608545424, 1.713895544728973, -1.3486753247416492, 1.6349432840550544, 1.0353768597529536, 1.1262475904043665, 0.9451828437268329, 1.971636649494429, 1.5627825164574909, 2.3028741887989463, 0.9362278395759613, 1.15734125430001, 0.9561944806303904, 1.1926565441379904, 1.483451104639665, 0.9324220516403109, 0.9174229189504898, 0.14997687804539875, 1.8516387507276082, 1.9614781440159954, 1.5263512589444246, 1.7852747777780542, 1.4362387491349857, 1.4315270414939074, 1.579586976557048, 2.572177858824911, 2.241830464404638, 1.12710925556341, 1.8340771065352703, -0.3891648338764542, 1.213291884301719, 1.132271657690822, 1.3400121945646235, 1.6841132528635907, 0.7343384562645178, 0.8462916982187453, 0.25538981837982244, -0.24933280437780028, 0.8977411943368855, 4.67766602180561, 0.557947092445398, -3.1023151466076624, 0.0980365418337727, 0.9229486873613995, 1.1115281538097972, 5.830704400801396, 1.6058245140712561, -0.4762244627252895, 2.329814286784221, -1.9758433148908014, 0.7528474493871272, 1.29992321854606, 3.3296359888868494, 1.8111588147631459, 1.4650534714237462, 0.057123637923836734, 0.6858658914151735, 0.648223587842673, -0.05181782736953719, 1.4064694783622051, 1.7846878836767066, 0.30846364260623454, -0.2765562928628197, -2.873999602373539, 0.34328898881746084, 1.6536479683877159, 1.1797285676199833, -0.692968651323179, 0.30163164249155994, 0.9478503576529391, -0.6980672450101023, 0.21056626832464828, 0.12208512381855777, -0.04011689286212941, -0.2773706515662343, -0.010843254573475819, -1.347817377426441, 1.2199071167934312, 1.7853705974693495, 1.38264762180046, 1.1120174750026555, 0.9540034348788907, 0.27368530666132457, 0.9364747359744362, 0.8466811907425246, -4.085677024378359, 0.8741413273123841, -2.733241200591693, 1.0763415251872424, 0.8588516714098612, 0.2840159960134045, -0.4069782614012115, 1.0069449115488158, 0.8356682654105235, 1.7407327897094145, 0.47300613515739653, 0.7615131699988864, 0.3324930040188928, 0.7397683023367095, 1.134808383903477, -0.16407126572238592, 0.5562922450063006, 0.8727510781442895, 0.9080263026827439, 0.7472071347031398, 0.4459052092426682, -0.23573994274354285, 0.9937328764549309, -16.758826239866156, 1.1457239906282046, 1.3110052172015823, -1.0964358476705471, 0.5711312726271566, 0.9018585127397999, 1.5331823974939354, 1.1391538695268875, 0.9673341653944209, -3.32725571287962, 3.940445075887852, 0.3075665147773954, -0.09801876131539569, 1.6672425769637969, 1.319890757741848, 1.4439535800301349, 1.6949403115632014, 1.5075529958350686, 0.9864384442187911, 2.741649030257463, 1.5348257773879797, 0.6681444540942847, 0.5027287943967522, 0.8050795649424006, 1.5277667793336074, 1.7693216176198199, 0.8450401035757292, 0.727257560630227, 1.4926469070969561, 1.4148121980656867, 3.2148907013745394, 1.5733322561544978, 0.7355487904248347, 0.4020227856662396, 0.2642900031297416, 1.4323684933392995, -0.7823312759590378, -0.8032649226651444, -0.09236151369697806, 0.46660477458993, 0.9331731592245925, 0.43907270683806887, 0.7470620420168064, 1.9148665292897034, -1.5024684003663207, 1.0124870554322047, -2.5767694098075893, 0.41083599231785345, 0.6213071225514945, 0.6465937042470782, 0.6894702181336392, -0.07421608757387313, 1.3534570087560287, 0.7202153881196857, 0.9848106991471084, -2.529855485077686, -2.2997220803549814, 0.90243677869816, 0.6134469919644303, -0.15578204028191242, 0.8737739258599088, 0.6706175997093252, 2.0223040066569657, 0.5964878726390401, -3.8106257848413385, 1.2675634425751805, -0.14868980579006055, -0.13784993967816103, 0.8283406580434102, 1.3365713932375567, -0.36303498742895046, 1.0642490660046076, -2.8384674670080337, 2.6832150747593624, 1.0316742287360503, 0.8575521891334685, 0.47687945068178006, -0.6835692269867377, 0.6478391858933236, 1.2123360948129223, 2.627777415549332, 1.3058747171720995, 0.7536612140298921, 1.4798694881442624, 1.4896506469074193, 1.8463882918090664, 0.6654173977253802, 0.946566215724678, -0.28145349705199885, 1.0241443769129823, -0.410562829392208, 1.0552892769367153, 1.0931550280237488, -0.9339422147734789, 0.1009087976374167, 0.6992184765302788, -0.17859605446066976, 0.8352277042013119, 2.0800312067022353, 0.7599282875492531, 0.4281180601239002, 1.8029563455304987, 1.167692522650469, 1.8810968320639845, -0.26306102501976003, 0.30937348411387633, -5.389813661994381, -0.1516815467543882, -28.470564433570164, -0.011965276521793494, -1.0786246809491344, 0.7081328346870577, 0.9027491490392318, 0.49518703447057866, 0.8097626853793591, -0.007116341602157322, 0.21863815159865818, -2.5145146862557564, 1.5009994029044496, 0.8687918521036467, 1.4484580120213293, 1.6390700567024699, 1.5622412333565323, -0.6780431624470048, 0.6975487560815774, 1.7135952506914192, 1.0915822693519814, -1.2545677326185865, 1.8977569270501977, 1.5590850413623545, -1.4844944639186577, 1.333649884786905, 0.5011998999541666, 1.9031194614536895, 0.681925136127289, -0.5668850098927225, 1.0370037843085853, 0.5681428546657503, -19.436317221916138, 1.175685612251656, -0.007092865118647512, 1.1016826909621367, 0.8666131788472041, 1.2644477490014039, 0.5746351319468381, 0.8907456466645733, 1.46850176213602, -0.21640494660045276, 0.6343170968395174, -0.3667686506655454, -0.058553078381550355, 0.9600458620713345, 0.8325206463543021, 1.273619949685997, 0.3571078499470578, 1.168048021382944, -3.6423798028099004, 1.1586309822296716, -0.2006324090928008, 0.7599282875492531, -1.6331048370418477, 0.7447518985992483, 1.0797917999617106, 0.37527291053477574, 1.3994962824242163, 0.15383355816181035, -0.953350160009189, 0.6723423162261798, 0.656375874489844, 0.43023066936284715, 1.364310033313447, 1.6142099567644441, 0.547713988729898, -2.2265024127014232, 0.299387226606433, -0.5997702454743725, -0.6858208689703468, -0.4587383715305765, -0.16479236194995464, 1.2374994047536598, 0.7541667080477159, 1.189403909305193, -0.5921563450635604, 0.1312784396064887, -0.5656151591044983, 0.7899823504136567, -0.21599435418312504, 0.3174245604882926, -1.7107466927974615, -0.46619245645448026, 1.4539536816354193, 0.3671637808620297, 0.6929494557905879, 1.1053149417622, -0.9156016866113156, -0.622131717466114, -0.6760340804822106, -1.254943597229688, 0.1699896727321211, 0.8254505515161046, 1.105495624666446, -0.13069435509139796, 0.11767106592999782, 1.3578819792789918, 0.9850184547783442, 0.5350264353244514, 0.7599282875492531, 1.6733683773711288, 0.5913923271396564, -2.2458763950709892, -0.5305378488632733, 1.1406239764506754, 1.2097457490254415, 0.9145285097496069, -1.575935163627465, -0.6449375448657587, 0.54193868188731, 0.500353859461204, -0.37437051125385346, -2.308013632074109, -0.7536411070558442, 0.1699896727321211, 0.9642853909511354, 0.35375126443324934, 0.9004609044484592, 0.7106519058650345, 0.1699896727321211, 1.6287036330598161, 1.4954365121639208, 0.08586666319333994, 0.49083926213405804, 0.9323950717825761, 1.7442154651736546, 0.5250011761288241, -1.6906256205571129, 0.7698344258433992, 0.8137621916264471, 0.8362935958318612, -0.3067301350145125, 1.1000194786820596, 1.1697083451799026, 0.5008202730539506, 0.6710161637636469, 0.523029175277662, 0.8651527612013907, 1.7407327897094145, 1.168814263087803, -2.7275069170651083, 1.6907936415524454, -0.925356904298955, -0.25993493328264583, -0.563086438726199, 0.67424793341068, 0.6288936309672446, 2.0588942345798205, 0.17485433982646612, 2.2511885101449707, 2.4569592913016893, 1.769429677696568, 1.6450575609417084, -0.12809345675148756, 1.746278547764081, -1.4643258168147784, 1.3003862553763224, 1.1447874367423927, 0.630385874102216, 0.28527088746063234, 0.5171397194262929, -1.432391878397833, 1.6904607396235933, 1.2245081371960715, 0.6379640180268311, 0.9732381910708882, 2.0318766003695963, 0.08890472868642725, 1.0908173377259631, 0.2920175669805214, -0.9978813640492167, 1.0590259627525733, -0.060864428598717935, -0.5994311664939911, 0.8304611131231175, 1.4866016826290036, -0.9390424765435997, -6.047300755744695, 1.0242231086092826, 0.9786198579697327, 0.818218006115816, 0.1219644000233415, -2.4027945996008744, 1.1721947622371733, 1.3050447577529223, -0.3471436254714729, -3.0832943433587956, -0.7518671617095006, -5.101914656025779, 0.9943463145006443, 0.3700950707277839, -2.0797787013908824, 0.9203359544781319, 0.49469970996834434, -2.06149475746399, 0.776415516957245, 1.255622384165857, -2.220398276631372, 1.3306126289782965, 1.4806971168370453, 0.7443288264611992, 0.7699019705023687, 1.383567788990246, 0.4370002418513145, -2.9312185349727073, 0.3699961668081423, -2.002056471655153, -0.4518009647492346, -1.3210164562109348, -0.21073618394831908, 1.3399208627307748, 0.7626014240764442, 0.8972932400093867, -6.701052857968103, 2.005011013744587, 2.2821835101137675, -0.04598119670161152, 0.22463423430810944, -3.0933713986022444, 1.8024429275670324, 0.5884614344852792, 1.6661092324494582, -19.53906559294117, 0.07362712771840291, 1.1644200290811673, 0.9870705113143449, 1.3300327634894473, 0.8539736461203337, 1.1046172602266342, 1.2759934139248237, 0.6811209088731569, 1.412064895586052, -8.242819160327759, 1.3140444230978983, 1.0682228030590746, 0.4808697760939509, 0.5856094755728514, 1.1693575866789199, 0.6016420826765808, 1.6733378930876384, 0.7050491958408333, 1.4073790556406383, 0.42495253065029814, 0.937830069338999, 1.0097479545461292, -3.2028396850843914, -0.6381897357982034, 1.856211451344879, 0.7515898574304495, 0.6977216329170214, 0.7599282875492531, 0.8200751668047128, -0.15132678395013907, 2.381222526091257, 2.368857497309501, -0.9133734491265072, 0.5242845891381273, 1.2859097308871394, 2.108979841298981, 1.573889104570871, 1.0680751759583578, 0.46402406941067376, -0.29108058728126673, 1.2915541891574858, 0.8625079429414422, 0.3062093888158296, -0.8677902613573881, 1.0219738617681382, 0.6056319332691369, 0.7039187274240148, -0.8958300810368006, -0.10960244789184934, 0.011355234649161883, 1.879912239577797, 0.8573616477072955, -16.391638544125048, 2.2311397962399284, 0.9810911880519049, 1.0003866303500344, 0.5945968274933805, -36.40161826203596, 0.18718068300615806, 0.9672853388087104, 0.16391308485178513, -0.64109590568517, 1.2806945130384837, 0.6985467083942707, 0.7811897523812059, -3.657839778260094, 1.5034369158839576, 1.2119126195046777, -3.238947957534888, 1.3051141234918129, 0.6914490749769953, 0.9135407832050367, 0.5641265645230462, 0.3792357291005956, 1.2826560923659298, -0.35382168527349855, -1.319988765617049, 1.11895451406384, -1.2888340265165985, 1.4799985823866977, 0.3031920150886612, 0.6213254728613151, 0.1744606013567316, 0.6220690470997641, -0.2051559662516527, 0.2922288291547763, 0.00280326913545495, 0.24192175503307695, 0.9036456003802676, -0.7609474746118995, 0.2974936958003541, 0.1540909247721501, -0.4688103646271381, -2.2442211169980335, -1.1361837657634275, 1.2791609786281588, 0.7047514219927301, -0.3450269966660142, 1.0816533885235988, -0.11940084705223833, 0.03444707058614957, -4.1685805531167475, -1.2018678148253619, 0.773990996562317, 0.7636132076822774, 0.47418576981201144, -0.06299862889656052, 1.0631046322003832, -2.8815561429198135, -0.2687911450128999, -1.0088716144560255, -0.3660938097667368, 0.0519352232266278, 0.20059836221985666, 1.0543080699099072, -0.6042949966516142, -3.6265191656930265, -0.1765949955081483, -0.7838929244233781, 0.1871807893531761, 1.4123275652037521, 1.2054506015171518, 0.5874405616954566, 0.8996164792173171, -0.38168826402117506, -1.632018700788011, 0.46760192452170046, 0.5071301612316298, 1.0693352238626745, -1.3052793693520688, 0.7377665048179999, 1.3943482806931198, 1.472198795184311, 1.5607237981357625, 1.6340047535375477, 0.00032170235559367266, 1.2058812566086397, 1.2155014938838633, 0.6153932409283781, 0.1699896727321211, -1.555688465966667, 0.7288354701276826, 0.5574554005044057, -0.3778284824149031, -0.18138686245422, 0.3759335538362557, 0.19564958099401908, 0.2822032055498922, 0.4959830776710432, 0.9483690085865586, 0.7807183055545471, 0.19636676587019658, 0.6425073268379865, 1.0802330742426072, 1.4891206244468276, 0.7868675284266454, 0.8261035806490435, -0.1034580977513955, 0.6148426395738257, -2.2048459546921064, 1.1546835521058283, 0.5490386987492489, 0.7261477868918278, 0.9902267197150311, 0.9400592594131453, 0.6472520524542433, -0.4182073271569541, 1.2076566085544707, 0.23983440121012642, 0.7599282875492531, -0.38960356148281117, -1.027399323167022, -0.9029643667790722, 0.6193907879860451, -0.4030725118074192, 1.8294346267249855, 1.4833849083472161, -0.7238479147027858, -0.34696668270513265, 1.0284132642671306, -0.1990717903602023, 0.35853787016400246, 0.7087289320562525, -3.609015382864843, 1.0559850635412698, 1.8139965283784083, 0.40103189420100643, 0.6517838509845615, -0.03742040129329774, 0.3462607396848577, 0.43562559600407214, -0.8016096391823392, 0.5861265157474345, 1.234553568330868, 1.268406335598419, -4.124107534460526, 0.5636013084281637, 0.4292118292640935, -0.07720334854903119, 0.8648526939002696, 0.5844373780323208, -3.125189741917385, 0.503613613593133, 0.1309481660765156, -0.18328003385063543, 0.5546630093000592, 1.1905161974048886, 0.9390236002846822, 0.38123763152309753, -3.6470577506749753, 0.5323654025840108, 0.7564913981150957, 0.27877611240434097, 0.8483552724739443, 0.8838250287112897, -0.578628053983962, 0.7749909829922415, -5.4580070712827595, 0.7609900004684143, 0.08062355701121479, 0.6045099564766145, 1.0675614128340352, 8.946872745776117, -0.2362755119300026, 0.6599483980819999, -1.14378586112731, 1.7124837024769157, -0.6006351206220605, 0.30044729565453965, 0.6939879737287379, -1.6374163676557572, 0.8977065744289326, 0.9839152579892947, 1.0342990596863673, 1.1344353855745535, 1.3203241541323107, -0.37347571238989097, 0.8720722025241232, 1.5625215915322672, -0.09109718649854584, 1.3548293089683827, -0.004768091982705475, 1.1512902686784996, -1.5132992022269027, -0.28727892373375097, -4.010178725228049, -3.383227011325606, 0.3825060474488028, -0.12351176426595686, 0.5546630093000592, 1.1272532162760727, 1.0131140018790414, 0.7257930972569032, 0.52358264651077, -0.06922346186995912, 0.4063436050121584, -1.2847499245819674, -0.8927077470874006, 0.3734762927023409, 0.7689549184821695, 0.11181265323001091, -1.0826212464344216, 0.19016479159047775, 0.1665053218701101, -0.047421002853010846, 1.1802942048743983, -0.0009339272376866214, 1.2392226280839207, -0.9718406429977783, 1.5605395166044824, 1.675772667549952, 0.9419854761241032, 1.8800037478214882, 0.29678358102175667, 1.7222592176787905, 0.6114847704765359, -0.37781626321798234, 1.1002168938898167, 1.1593528824155028, 1.4177504498038038, 0.9746202590142089, 0.5439383301895595, -0.6404474038052284, 1.057815723570748, 1.313633232123898, 0.6761339341317454, 1.7156048715225944, 0.3552286614869953, 1.7693764688349554, -1.983243489835384, 1.6147857129734107, 1.635247306053933, 0.6028867135066358, 0.505937622339142, 1.2067327963668222, 1.2735490435849204, 0.816598538521119, 1.8254040323315475, 1.1850244735921536, -0.008712689937315665, 1.4744507972846823, 1.026272635231578, 0.9269393402416273, 0.25278337092782166, 1.2141486348830084, 1.7120318253327815, -1.6350453642623874, -0.4910654627445783, 0.45823552069468176, 0.635475060350672, -0.349304195233176, 0.6885386155255039, 1.247698239629298, 1.2657783097755357, 1.4790300814014037, -2.6442415445552836, -0.047607161322487684, 1.4053348250303555, 0.5453886372521408, 1.4800784314553515, 1.1396386258186857, 1.243436487544954, 0.6805608037431704, -2.4246473989074224, 0.0885875984927453, 0.5286423984015853, -0.2431214512608951, 1.526041181087859, 0.08994965287639675, 1.0641158811828104, 0.7534783328948979, 0.7952202418889213, 0.4286171918929026, -0.2697248286400703, 1.37061169562734, 1.3898945625278438, 0.4641182499276334, -1.5593056670747836, 1.106131554041556, 0.6408754866936557, -4.012572482420502, 1.477333882177019, 1.1511891167555566, -0.4422921037194296, 1.049676986054379, 1.498502207372385, 1.0906081222546669, -2.080565861687342, 0.1811560160884124, -0.5362957527421658, 0.3495540606964712, 0.919043330717608, -0.6416912279764674, -1.429397068398406, 0.827793023208805, -0.15125927738245928, 0.9007978293388466, 0.9348299071468804, 0.5663094089093943, 0.1546955116786105, 1.5990345998704016, 1.1498213709883187, 1.3463724216335593, -0.007660098944541255, 0.4844390465867717, 0.0452693547984184, -4.942667007448967, -0.49624987841783785, 1.1087515931164924, -1.6508611832914775, -1.0371503706186336, -1.8426965092329908, -0.8687133621859092, 0.15428388477485155, 0.6671975831488887, -0.8415769665371492, 0.5945580139981451, 0.706954225877204, 1.979506549607062, 0.5059383218254225, 1.273237081475951, 0.9345818122776457, 1.4230617524073377, 0.8978452919392134, 1.6435806345549524, 1.0099074793414062, 0.4503746304854799, 0.7904170110250205, 1.3561973993422358, 1.1646514620092536, 0.9799849675323745, 1.1972524096204773, 1.326796559806772, 0.8513341378863031, 1.4118869492709687, 1.072616744561331, 0.03339990017987052, 0.48663942597157106, 0.4824816824595888, 1.0770467031514228, 0.6162297356318673, 0.5862281050873481, 0.3003095686060245, -1.220855864873593, -2.6173309672710627, 2.2896482260089903, 0.18503238070723244, -0.4001768701656827, 0.21542991003256404, 0.05054460205432998, 0.8368538534488016, 1.0831705594249217, 0.628534250196151, 0.5404948732442274, 0.4864297947344275, 0.521586450309132, 1.5048971668890427, 1.0971215055752541, 1.0642597493948374, 1.56036813935171, 1.2324189333146105, 0.376222269561255, 1.0252565395925646, -0.4665426430492935, -0.009930550526501575, 0.296301446717531, 1.040101946429252, 0.6576254679101053, -0.999180467728917, -0.3581440768103955, -1.0855830810976976, -0.6058623821144268, 1.193879186109463, 1.430587137854989, -0.4337585227829708, 1.0064456006992064, 1.6455949317912673, 1.3737445373306392, 0.26734282863801895, 1.3870869485038326, 1.242160043940993, -26.02321254111769, 0.4335019663547306, 0.12662032467755302, 0.24051770256711527, -0.5741033700239284, -0.6681183325149093, 1.0507835710061084, 0.9548567659426894, 0.17622386972981394, -0.009214881813357861, -0.0062611807865131375, 1.2712743622442633, -3.097807236786918, -1.3435273917195565, 1.7704325684583162, 0.3094276815362994, -0.48701692531589347, 0.501441140809366, -0.32984442804390335, 0.37527291053477574, -1.3140052826920485, 0.20055824828139387, 1.0399305353103998, 1.3698118552868916, 1.0532109038295838, 0.5782263688848615, 0.5276222863089839, 1.1495676063828628, -0.07606495318271303, -2.476084065244011, 1.193018682306978, 1.105820612952529, 0.580780612494269, 0.8399613867978836, -3.0486245616968395, -0.02559643526249855, -1.7406862960751217, 0.2567308478889734, 0.511727247796092, 0.9291858216278126, 0.2349210649733869, -1.427497155967345, 1.159296961443139, 0.9284471006422603, -0.6165045579012204, 0.500273002696658, 0.9762546132582933, 5.313778969018712, 2.551818096074139, 1.7311248385184603, -0.21983879162240447, -0.26692865400727794, 1.0539974152378673, 1.7863166764451825, 0.953394637815999, 2.086816779528772, -3.21488379524982, 0.28129784046323797, -0.08049785953415836, 0.4760735669911059, -1.1972725098973263, 0.27170690173698553, -3.9015383270462105, 1.463710809273538, 0.17804549283456875, 0.9496213061082747, 0.504749062375855, -0.23397893398988753, -0.08180961335790196, -2.690085424391034, 1.0854184668104288, 1.6553782587016996, 0.18761932108418253, 0.7147128053721711, 0.6706897933694439, 0.295008120053869, 0.4423163011656094, 0.759083800805671, 0.5134911353300168, -0.5856551872528689, -0.12661188128859194, 0.5658726516393208, 0.8561660243638264, 0.40913559125035825, -0.8016096391823392, 0.993045338715098, 1.7531206880466654, 1.4176092120602466, 1.6426287592795568, 1.5248569018657978, 1.7890982258634505, -0.01712820933310258, -0.8415895619004422, -0.7819968959610724, 0.15302593968134287, -17.473593663037665, 0.35212494652623166, -0.1421833450207627, -0.9001514989828384, 0.21567526840531556, 0.7699743603064324, 0.7465907077321736, 0.9303169780014974, 0.6268011944240763, -1.370514007587637, 0.9863936201784435, -3.170082906373053, -0.3524746239748219, 2.2043521890968796, 1.6672318468135252, 1.0866376970558234, 1.0818175074257317, 1.3250293718721735, 1.4010668022616581, 0.9967649902266286, 1.126817127518636, 1.3834937629536854, 1.1383541938486144, -0.8280377497502921, -1.2134743672745183, -0.6397859227456203, 0.38590303891984223, 1.0791164495726624, -2.467789615300841, 1.6089184077412086, 0.7847995160140245, 0.5239024150434439, 0.12677851710825616, 1.6044483163086414, 1.0840240156256062, -2.4521822113603298, 0.8340755125098238, 0.5239773238921831, 1.136872417047132, 0.5809795549963012, -1.785378043481495, 1.5917699206045188, 1.2156702993666144, 1.0049427276974887, 1.0049893301778376, 0.6121313326367009, 0.4551449132612554, -1.9791356411873633, 1.0174217361430904, 0.2137792525923126, 0.8403237702987946, 0.7134942628991175, 0.3107844908746078, 0.082691299084523, 0.20142841492228056, -0.19152328834069957, 1.5853677885433595, 0.38762195392179766, 1.4800069957028605, 1.1695208311214307, 1.7475152916535197, 1.4268206190173685, 1.7899488963866015, 1.1671112382846398, 1.3763982080315567, 0.5050899691712062, 1.8167274968575495, 2.2544541272305523, -0.7281952720748389, -2.380854243577697, 0.4710297220526084, -1.64260850268758, 0.9651947589757549, 0.5523048423020992, 0.3179081896709046, 0.5747060853353804, -0.23047502397324202, 0.08929543501563908, 0.03756351079135453, 0.850176805867249, 1.5872708635181514, 1.188033402723443, -2.8235415478448003, -2.9463907307552417, 0.36557886010622875, 0.3745257659954875, -1.73040214919688, 0.46909769910341226, 0.2834501957252346, 0.3553695796152945, 0.9388951950090324, 1.326796559806772, 1.6822196657888702, 0.8513341378863031, 1.331239069157877, 1.7501486843359941, 1.3086497327859312, 2.073160871084287, 1.0320161401976544, 0.9440050881701071, -2.8123531637486043, 0.7609676277476342, 1.9350848828399483, 0.886664205311445, 1.3253724372748648, 0.9723800467479291, 0.6737897876513884, 0.34964403845614833, -0.1668608816683462, -0.39930151941800307, 0.6833228833102475, 0.5675001356240308, 0.08256836859745022, 0.786348057291802, -4.185083320845305, 0.8764037428279364, -3.6393847120388485, 0.5398764789359279, 1.5565687390279408, 1.2166766846862374, -0.48006698374152146, -0.751456956863897, -1.5264259801580315, -0.5823081464677339, -0.11895455019621129, 1.066312197472513, 0.3004244975702669, -0.6015444892097226, -2.6120355500418855, 1.7197945995052057, 1.4075128131160863, -0.02523371734232585, 1.3607359000752963, 1.273672697464729, 0.9090257610283055, 1.589786557148315, 1.0755479337051828, -2.096744756155759, 1.08290117391648, 2.1621147801336362, 0.9875215634110284, 1.847689767450089, 1.2321984724914916, 1.713619176595652, 0.9376628846209609, 1.3649972605496068, -6.178435466999207, 1.3004282320398801, 2.0480747668016823, 1.5563746302042603, 1.8132413921436112, 1.137002599770107, 0.9942799445812214, 1.1395236461813265, 1.0211140574180806, -14.092762460683371, 1.6091146816654103, -0.029340774807044695, 1.908921102750713, 0.924979598590506, -2.2217896672453916, -0.8910330321301015, 0.9346702943532132, 1.4398387148200424, -3.774161611824859, 0.18076123685065124, -1.0354556872056666, -0.9690545114560146, 1.5105326325192365, 1.371964868580842, 0.5546630093000592, 1.3305326314293373, -1.3014295926502855, 0.8803924948492524, 0.9042514387774214, -0.2272834763373587, 1.2996179578677187, 1.6521448311955405, 0.7780287639778977, 0.7724816142556432, 1.1455971460321435, 0.8773898583121039, 1.2356149837940325, -0.3691448968627383, 1.754086479871744, 1.221749923480836, 1.6799388045261539, 0.3313580818130267, 1.0730394576294993, 1.6728650982650601, -2.076290757047636, -0.3527226903549089, -0.9129527479903073, -1.1874979367728862, 0.0001413576330044075, -0.12771016686032013, 1.3623709812911617, -1.177503570446194, -3.1985618784213927, 0.706253662782736, -1.115987314707653, -1.3746721802677744, -4.050182488836065, -7.153039147738072, 1.1163646934259355, 0.5920182453592091, 1.2857446248630822, 0.8603101869076885, 0.9187803688658922, 0.968740076483509, 0.6565715229051549, 0.5723904290464237, 0.4640211588438131, 0.7095962260850067, -0.4479555315971354, 0.49717482858791184, 0.9452489918821452, 0.9119951283663533, 1.188293229707188, 1.358629459061003, -0.4921142876443584, 0.26275784605631153, 0.3841210459530903, -0.17372283811799913, -6.7222405846791595, -1.8525439077019772, -0.9031002164055616, -2.587813363221573, 0.5672221984850075, 0.20478597340907803, 0.8959234029992765, 1.3494988638908876, 0.9941418960722782, -0.1880610170717779, -0.41568731804684855, 1.1797382980645057, -0.18945728678243998, 0.6641244307745066, 0.43704982200969966, 0.6196966923306058, -0.8440502035561818, 2.3419048828557023, -0.14154655382518713, -0.22692688083718907, 1.0612804548093722, 0.9666487391905368, 0.660826471485666, 0.6006972801828359, -1.9064783432724515, -0.07662593578651536, 0.38396017401957416, 0.03516006781446379, -0.07436503214409831, -0.6504251226539621, 0.2867141205334579, -0.17703958991852928, 0.401490997448263, -0.8016096391823392, 0.9470212948444893, 0.5294098904319043, -0.6442308964569509, -11.003184684336407, 1.4033747995270345, 3.2094243319538904, 0.7456097839692989, 1.008522588468822, 1.6394795486963378, 1.5605424991526027, 0.4858118078601257, 1.2894007233953875, 0.9931519480617388, 0.7911513567608008, 1.1386652562030941, 1.378212217835552, 0.9683131011471374, -0.501906803789889, -1.3432507332124697, -4.126228539753703, -0.2544234726471186, -1.8425660986847612, 0.8156546635060826, 0.40917614395026336, -0.20846814508278513, 0.3868934156379068, -0.14607055688896206, 3.0104627013741387, 0.8603005868738984, 1.1584108399075488, 0.8147439621940826, 0.8521976554769458, 0.42647172817915857, -1.9702903963316145, 0.8004497441048167, 0.9004059403670542, 0.6912800034142937, 1.0629826566755156, 0.5020104318512648, 0.285555962151388, -0.187946836026436, 2.0886576840341466, 0.9244693947557397, 0.31144880768256356, 6.908744560600728, 1.2599897494508585, 1.5278297135507855, 1.0038797745209456, 0.5013657472060808, 1.4054706607519778, 0.7989250131903182, -3.555967716643277, 1.6618850988390603, 0.862649323069988, 0.34343371006176737, 0.9690802567328283, -0.6204995310232643, 0.7538302857547764, 1.80959955618428, 0.3333101079039257, 1.5655527397272373, 1.7130917600165756, -7.836912106026641, 0.7195949027028042, 1.862952317717604, 0.8148390020929671, 0.571558913893249, 0.9862911394588482, 0.30593316779256796, 0.749662066780431, 1.082661754455897, 2.0136547829577522, -0.49398275192310803, 1.6377147100305507, 1.4552135572242328, 1.2552409382379077, 0.18462460496182811, -1.0308964199126167, 0.0183228991943635, -6.630467027494047, 0.008175995323731922, -0.04844589006507713, 1.0101028267153471, 0.0860197292395006, -4.683962981294688, 0.7028207014912125, -0.7757724784714584, 1.3993635684598338, -0.9323706180183519, 0.017141358954771452, 1.4684284918020982, 0.2186422187295541, 1.141164921031647, -2.5828215083546517, 1.4234711573737935, 1.299163393470134, -1.4965287402175649, 0.6030930349087792, -0.3109346306240034, 0.17581706573979802, 0.28353840113118545, 0.09037293541165334, -0.053475274345364004, 1.3474182639012546, -0.7695620344240193, 0.706168446211251, 0.27723358139858356, 1.2420618534058445, -7.336881652706903, 1.2920940075234173, 1.3865825563231802, 1.5442866088592158, 1.0199106579724466, 0.520101259777231, -1.1314409186639673, -0.33118569909671813, -5.060433800103748, 0.17061758935573793, 0.7827865612631403, -0.281739012031728, 0.7509298117123938, 0.31578815030872454, 2.1942133179561956, 0.9452489918821452, -4.668211949556271, -0.013175716750093519, 0.22862103425430494, 0.1946272223270869, -0.19067628110158105, 1.4574342109987402, 0.7508721422451169, -0.9218939034758084, 0.9397921225634712, 0.2978460163024672, 0.4976605599522693, -0.6336663536354703, 0.9332390487479095, -0.021184906636940587, -1.4301462323610628, 1.114850716686791, 0.5142572375371618, 0.29247415285048095, -1.9691260405155389, 0.7134318169898769, 0.6252252787211904, 0.2119021227163222, -1.1039340074005688, 0.23824538194501174, -1.1741383782910737, 0.31920557748463496, 0.12228941594735918, 0.11829746287468271, 1.3535063818016038, -0.13181835101448794, 0.7616492721153659, -0.3778209987177591, 1.4124455406777776, 1.2556014309884331, -0.6154852287081445, -3.0376610878559855, 0.06171557173634032, -4.474045167874764, -0.00985046220111066, -1.0401156153585418, 0.1199505059558896, -0.11277004330745691, 0.2492240658148233, 1.0250677485957924, -0.5086473733003062, 1.8929610797847076, -3.912155815599707, -1.768349704671507, 1.2730895838439846, 1.0408655408067549, 0.524899814060284, 1.1849976464249758, 0.5005103987793926, -0.5575169639529179, -0.3064802574262665, 0.9604881658901301, -1.2980789314469983, 0.8271377883419017, 0.3691205555643, 1.3113297171019254, -1.1881881690673453, -0.5684121081430616, 0.29833615642871947, 0.38700889299099295, 0.40171330942255984, 0.009324054147597877, 1.337452733715442, -0.5342428480861853, 0.12478631918110776, -2.973423393310793, 0.9798916556554962, 0.5111798383027673, -0.9058749414710509, -0.6860561676741558, 0.7951050350144725, 1.5260709080177424, 1.426175924079555, 1.5198655559471541, 1.2688610571196342, 1.3720912726530365, 1.7834248400548907, -0.9074658573821056, 1.3610408452700609, 1.5456045901386464, 1.1179198245051596, -0.8208168212965031, 0.9683208917529987, -1.6362757367836096, 1.2962531904749561, 0.9441505652122938, 0.1137094106524196, 1.312798239420356, 0.42803429792062675, 0.8156344239046041, 0.9509449901737342, 0.23038797304975459, 0.7013375880714764, 0.7510681262627834, -0.18719729385543224, -1.2489852038496125, -0.720398748205246, -1.6151437812581741, 1.5130027267279913, 0.9490885318798422, 0.5250174923124216, 1.0781412283070269, 1.4994861976323797, -0.007338910944512195, 1.7059537442805421, 0.8475361559544913, 0.16492484548149816, -0.18497014402548748, 0.7039706105768825, 0.9579153754435358, 0.6084694362491623, -1.8825104737813747, 0.3542020935236655, 0.946179592545764, 0.6054864319644478, 0.7665537262185638, 1.014663623303108, 0.9926640164035276, 1.2261838617239167, -1.7056604050586128, 0.652469962660333, 0.4004520079960987, -0.28998416959576834, 1.2943240016140474, 0.23037489163708438, 1.4453871056315801, 0.1495478003872688, 0.39748892253023405, 0.35402819084287296, -0.56410423047669, -0.8498587321442183, 0.19528918228612485, -0.29221400878511605, 0.8624244190215067, 1.0851494484968522, -0.15319441464640418, -0.816925507933349, 0.953948433702476, 1.1205497296951494, 1.3701821578320579, -0.3132317681230108, 1.6392586012091732, -0.6194354582901699, -1.1527694530417272, 1.1459388707186124, 0.4746620574060778, 1.0714790352715609, 1.0714615081329422, 1.3269849188094305, 0.11125171435945627, -0.2522522122234533, 1.1332425116593645, 0.8565994310692233, -0.002766926769413064, 0.4992059385607176, -0.6881584567260132, 0.5273042803231455, 0.2952080685261681, -0.20589841019534125, -0.7415837662823044, -7.0115747857365776, -0.14698444389190982, 1.7606154056723524, 0.9234471342510061, -0.2630492445923801, -4.1770520660356585, -1.8087351301447931, 0.3978160667422994, -0.37667476607195394, -2.9574296550425685, 1.3921339950005747, 0.8129550215830463, 1.053297435593499, 0.9842596379261728, 0.4134595773422215, 1.5799657092382773, 1.805046227111303, 0.7090911072245308, 1.3619640111836462, 0.12481355104384165, -0.33787138432309094, -3.0228459774120746, 0.3334300103433926, 0.8038791280637326, 1.0543890290043143, -0.5139385653966683, 1.4335205988770054, 0.6076896860487196, 1.6340925500056664, 1.6838813614060417, 1.3822584519670638, 0.8778657578762474, 0.34152949385074005, 1.900532149429142, 0.5986450013744204, 0.5546630093000592, 0.9017235667744162, -0.14277476122221658, 1.208917328957636, -0.08534335677097035, 0.25045993452512044, 1.0471763006978092, -0.43328364113063944, -0.5125081707146636, 0.018960999165725828, 0.7215635836666183, -0.0028209905326957996, 1.197276232158419, -0.9285629489042043, 0.3907215321598322, 0.4244865232748505, 0.4806211465146074, 0.06956369994514491, 1.329035916832596, 1.103575746755723, -0.6022463603573964, -1.4868191904200696, 0.24189613168405932, 0.022353708216898277, -0.2869777299484957, -1.270027474976156, -0.5904153311395004, 0.5690266373596692, 0.5767352934563038, -1.7611288660234217, -1.4638394744140977, -0.9661126167242061, -1.0110417866237484, 0.5756967160414004, 0.22743735944511828, 1.3468243094778647, -15.09740317137335, -1.6917524773590233, 0.1699896727321211, -1.1045846379874702, -0.15340554246197122, 0.057428733069759264, 1.3560139590717684, 1.4498254519574647, -0.7955756536850924, 0.8054853399225022, 1.7224476835932674, 0.5828325233844096, 0.8252345806840563, -1.1521703634074618, -0.015131275172065833, -1.0987746114020713, 1.0139824788444955, 0.8522923511576228, 1.2231999182190105, -0.6161989704599966, 1.3672028803569904, 1.1563711908025756, -0.7049427617676836, -0.6270537138495308, 0.5779328510667601, 0.23168341370693227, 0.6453759567941987, 1.3508956808982082, 0.1075598375305015, 0.6482064106712807, -0.13897065012683743, 1.0335926761832226, 0.9728063643344459, 1.760861529729111, 1.1937298150263476, 1.0479165856305503, 13.60903420686528, 1.4170223904587358, 2.3628721287545504, 1.4858120457908823, -0.43228017011657294, 0.46802602579270003, 0.25667552579935987, 0.6333352257578776, -2.6190721829829258, -0.5211185039153572, 0.8733029019673081, 0.6559453711386186, 0.40384125132924725, -0.01402233687999313, 1.1612041873863204, -0.016281635723405935, -0.07764309314104825, -0.4059214045583833, -0.34494774647115023, 0.6773407946423611, 0.19105739421858456, -0.04767200878914496, 0.5931515705092553, 0.9615200210344889, -6.541784320501969, -3.943419869122836, 1.2594706865319492, -0.7801632767011426, 0.38974566151128354, 1.4064584589297489, -0.6623424706371522, 1.5995408945066925, 1.3141133574859978, 0.8828324588399835, 1.0041537200343162, -0.19871600276622994, -1.5796745706670223, 1.418242870545561, 1.8802028023441313, 0.43651920310068537, -0.24290052214154753, -1.9818288368556398, -3.6103482344544022, -0.5176205831879349, -3.844253555607155, 1.7697616966397696, -0.13319406499813372, -3.0103233922557946, 0.6092693350154373, 0.15496496505331406, 0.05541242741462157, 1.3573514148326313, 1.3399674191108435, 0.9490949177360856, 0.44113662396801046, 1.0952580738582762, 0.002691742220503862, 0.03925869166819418, 0.8878549280550704, -4.8839473265510405, -1.0314918045119827, -0.5385276466327801, 0.8491295665092633, 0.1623848504642485, -0.6268836481261476, 1.7157653735184848, 1.4818115741953446, 1.363719244964041, 1.9712126682145972, 0.6917069603049602, -0.9800377314842535, -4.372636642759699, 1.5134806570493302, -0.22553844726408648, 0.2781140467776466, 1.007606297617942, -1.7001198643048179, 0.7900539239796084, 1.0710136453824493, 0.01129309278766924, 1.5096058365505733, 0.5404350429615271, 1.238820081153142, 0.7676219608161455, -2.0585965512822675, -3.8939380299945276, 1.7914235844921784, 2.5771316075165713, 0.7379120440638035, 1.104974519639998, -1.6526161344460186, -1.3376874863443697, 1.583730774186641, 1.5937237498343089, 2.1409080745444675, -0.6503253940127612, 2.1244290036153033, 1.2960424780456827, 0.9650854860385732, 0.44378741355021445, 1.3010547008323237, 0.8087908334078213, 1.5926000091494712, 1.6373570816720704, -1.1912245329258868, -1.6001526165518105, 0.8935302305276197, 1.8784462681223624, -0.21386554231244068, 0.07627099302273185, 0.2138547279105458, -13.870666966437089, 1.0122784146531814, -0.9477196523392045, -0.7867373187067187, 0.5475284388188532, 0.630993212960899, 0.26935040463804416, 1.3839758775541164, 0.5551142445930326, 0.7101057626353822, 1.2204765528477883, -0.052176693514061984, 0.8521971130208431, 0.722977124352944, 1.228734024582856, -0.3367688730713402, 0.6831995241841351, 0.7463226739608093, 0.8519474986535671, -1.7915722432148515, -0.6605038348073878, 1.0347416908008493, -0.20283782265048858, 5.834664266001443, 0.5434921247331038, -1.9866344075955251, 0.36437298437398424, 0.5546630093000592, 0.5546630093000592, -0.0005372637855708406, 0.9935630434134158, 0.4505064094395099, 2.1903079325361223, -1.1387887025393144, -0.28866319414742725, -1.6881679784065309, -3.529061258068495, -1.1543826398939523, 0.3298959038946042, -3.539691901031442, 0.1419332583527342, -1.203703473452043, 6.407696786075007, 4.138979410505593, 0.9805823873786955, -0.10143757741907936, 1.8513230132392082, 1.5177754486754438, -10.016556574417644, 0.7599282875492531, 0.7599282875492531, 0.7599282875492531, 0.40589657979684135, 0.7599282875492531, 1.7378752877951633, 0.9916155215226686, 0.3516514967897135, -0.41774673349466673, 1.6031059527922316, 1.1241204382059673, -0.11137839910259292, 0.7901379026149944, 1.6341106159690224, 0.9045082107393513, 1.8039544299464907, 1.1205605294384566, 0.9642925999719384, 1.914735416874742, -0.2302781354339045, 1.075073794408811, 2.132068061438451, 0.2453174162278026, 0.9766677648847842, -2.8106272883126193, -3.7431747307297645, 0.1699896727321211, 0.3242813794381616, 0.6246440993510086, 1.6915710705670708, 1.1371536552110169, 1.2549499815027159, 0.9569259878660594, 0.919909256335428, -2.4912482794202404, 1.2678653749697182, -0.16896627517753662, 0.44019752960024316, 1.0961841781404278, 0.9856307572751541, 1.0206595365657694, 1.889861509377066, -0.717553582459716, 0.39882217473860027, 1.69279124992456, 1.7075323508097187, 0.4085715318667176, 1.4408654512050403, 1.32226581925115, 1.3625189207611104, 0.7685388029985469, 1.1657959597125267, -0.6515176931829043, -0.7478607283021615, -0.2536044259622837, 1.4598918508712662, 0.9389380334998145, 1.600410704888207, 1.302167848119626, -0.03571871675827559, 0.3942893289287542, -0.24744555881553573, 1.6590494216972407, 1.2936255956577034, 0.28964625663667015, -0.41932024595539286, -2.537754674498181, 1.06320733809768, 1.3861711399510261, 0.805163472106739, 0.1322909508868582, -0.525920220594232, 0.7307489400206234, 0.8157929094865607, 0.659676613347243, 0.8335122308396217, 1.5591170873347964, 0.5537943836315863, 1.4828386854286506, 0.8597495486793815, 1.3670957099381498, 0.5777047651091473, 0.8558433452611623, 1.3783711479151715, 0.5808115465229017, 1.429533247453539, 2.052260562307594, 0.804020624647784, 0.44030266340843116, 0.5537975798708381, 0.17478277044400967, 0.6085496836670066, 1.4931203769709376, 0.9961508529667539, 2.1525591549624803, 1.3474318710804545, 1.4454559421917859, -1.5699210286126035, 1.2655934926591934, 0.9333444129490891, 0.8309361268100685, 1.5136575769975171, 1.3381215490832477, 1.5911405327148482, -0.29877701363412035, 1.8754842185775578, 1.2460058114742043, 1.0722619765279708, 1.3065831951575038, 1.006530424752569, 1.2677407436340902, 0.6861417369569227, -3.7656210070040603, 0.3091152007409832, 0.7021976503088296, -4.517820389322571, 1.1166744404312339, 1.0639372198798884, 0.7964014698812032, -3.223171418519188, 0.8463577207640084, 2.196271305036709, 1.7926254994583075, 1.3855583169896262, 0.8590263574161169, 2.0005756539855684, 1.2291915658727477, 0.6459322870079616, 0.6022787443573041, 0.6833327693474067, 1.7175408352907635, 1.6471774599657536, 2.067516927142303, 0.8258823955538959, 0.613852919592879, 1.8024378619574808, 1.9332545445877896, 1.2550272725189386, -1.2221240842627783, 0.9998641146896367, 0.8501597655792094, 0.26686550083175353, 1.3304261024921973, 1.4848958379662083, 1.1794940556717153, 1.5426900373761083, 1.1088242148704766, 1.845545044352957, 1.1675101951783893, -3.4617357249383383, 2.2892822447576653, 1.7113172139337143, 1.3609083757371603, 8.260909304136318, 1.988037142592818, 1.2472025266707638, 1.233308645196986, 1.6828466401223523, 1.3953154126229965, 2.9366531444472392, 0.7434819144746164, 1.5930640225967552, 1.8209229474374025, 1.0784045819373287, -2.405719849102333, 0.6977410731888772, 0.3586656240354119, 1.6404882275049453, 1.270794252805042, 1.0859088479757295, 1.7110264787154141, 2.4617599831062504, -7.533229586102374, 0.7506687381814235, -3.622610667340859, -1.329174718330557, 0.4063045455597497, -0.1888097711976632, 0.06169447019724261, 0.06339637500947012, -14.414854156584685, -1.1724584223315124, 1.417956064166783, 1.3322682696669343, 1.2805552231683397, 2.9972506891367647, 1.607959652029116, 1.6473495687920332, 1.2132375595150124, 0.056691019422750996, 1.9702313879195739, 1.164922867452637, -2.154127133932921, 0.8675368184343127, 4.61052355224502, 1.6078042271348005, -1.245740837068214, 0.533827927160634, 0.34533574111034915, 1.473588868761335, 0.7439839887829588, 1.3054522123660715, 0.8954642236801477, 0.31314730335683083, -0.4616765102605598, 1.9376544784751462, 1.0827923454090864, -2.7988991501213896, 0.9858641626945245, 1.133530838666969, -2.7295499277263415, 1.3909495282110589, 0.7028826091848498, -1.5560152909092497, 2.1351842467816664, 1.0993963623651113, 1.1558596105071952, 0.6265697701045672, -2.1879565754772132, 0.03342287812421452, 1.6159188857959739, 0.26486381015757277, 0.25529494958927046, 1.1133655141310603, -2.587403076911202, 0.2053018878345668, 1.3192857555943733, 1.0809518822960344, 2.408014890316765, 0.7021618032446751, 1.6350787248049572, -0.9347663405464043, 0.7439943420607167, 2.488844257918574, 2.076967791128121, 2.2733927442580724, 0.047347481192409585, 1.3242585652030923, -0.5524826047757566, -0.648586768420856, 0.49292971063572955, 0.09047231455883739, 0.9430453800245489, 0.7474194310897914, 0.2185694606942227, 1.798908662013236, 1.5483245247739497, 1.3639032955145318, 1.498941078265244, 1.6707069917485704, 1.0210428093269575, 1.7980298183794248, 1.6715634829090174, 1.2049443726077242, 0.6752744031931303, 1.2663105864260866, 1.7336898917024153, 0.032474538073953535, 1.4765544748312571, 1.5080587087709925, 0.5048032537164361, -0.4954255454754851, 1.564991965927747, 1.2027151076601712, 0.029769640404611858, 1.391283519746991, 0.5422058809208625, -0.6253769227198938, 1.400673376180831, -0.2484232974266339, -2.1058414751017906, 1.681324289952069, 1.3909355968521473, 0.558019103126204, -0.023465463688581045, -0.2319018571697076, -0.48497939337245566, 0.06402941021012867, 1.1090703865393936, 0.47039480607885764, 1.5332597981198186, 0.5904563800598166, 0.5998292385065224, -0.207297338331052, 0.17388073262822473, 0.7409918457560077, 0.9847120244694184, 0.8392490315209846, 1.7993941868930243, 0.6871156888580072, 1.65197447488857, -0.3550403003209933, 0.050230696584881496, 1.2679423671452286, 0.2855718500133335, 1.1780168672526563, 1.5285974876313961, 1.3441401974445506, 0.25110922603023167, 0.69131251977997, 0.5181568630152444, -0.5033085027812766, 1.074453117246474, 1.4093681340087847, -0.8612062721267751, 1.4249128856799167, 1.1257471336429459, 1.1184967030730864, -1.1408883604623747, -0.5552901033508001, 0.03094019516926705, 0.9143355482748637, 1.0130702857944784, -1.9611672530402677, -0.13863744728514418, 1.6209950064892327, 0.7005898962513011, 0.7599282875492531, -0.18217182272137936, 0.9043703154209985, -0.4057478487584282, -1.3700157529991408, 0.7547012799815701, 1.288769588549475, 1.1176336461076481, 1.0337382673452782, 0.03792801216667238, 1.2144407750065236, 1.709588352714285, 0.18672013368289012, 1.6677443306787072, 0.21523968579971497, 0.18656953579368604, 1.054364410686201, 1.3565671098767071, 0.010320237666158832, 0.0823043992497006, 0.08573826137560417, -0.6718923426850518, 0.4241729862683228, 0.5692894747739654, 1.0630675474153848, 1.9012870839506837, -0.16592847240753442, 0.9184624350092198, 0.9072468223737926, -5.205276250390472, -2.867590298535125, 0.8805124466514639, 0.231878782576329, 0.994735165878646, -0.9918475638431555, -0.730701140340881, 1.014010752466007, 1.360735775310281, 0.9511867771989831, 1.2281145243644798, -0.13110830990996583, 1.6118384952859532, 1.13707985559289, 1.1667361849306934, 0.21078930949721386, 0.07645386503876847, 0.1054043257129699, 0.17370104023451635, 0.2621006979874656, 1.4271271475699272, 0.7741607546433049, -0.06998168803069704, 0.5663755990086893, -0.14412855844254205, 1.0135205615015905, 1.4976950251055896, 2.7944015943445897, 1.451196858481558, 0.1211409987977567, -0.5125210479197688, 0.0565813420848503, -0.14749434706438921, 0.806512755548893, 0.5059305258394524, -0.904300474782267, 1.5531188226483112, 0.7604225836054045, 1.3683615020952198, 0.6683893470872057, 1.228510968823076, -0.008347638180433742, 1.1154225363966392, 0.943551857844528, 0.8768806954321157, -9.340647831898604, -5.3759064203142275, 1.1509213516802734, 1.142399302120819, 1.767329534130923, -0.05190969766604406, 1.2454911947870202, -9.057034875906146, 0.2546882399793565, 1.5598335843926776, 0.7371236957931091, 1.8559525369792842, 1.2778068169492014, 1.2668203447241173, 0.04271673734777923, -3.7485323742096095, 1.1653166558261105, 0.656063092926449, 1.4326286040072507, 1.6752548829395504, 1.9053349735760619, 1.4398474636291099, 0.319168880888609, 1.5350509297651258, 1.0874770006373595, 2.1591732106820944, -1.6159509832395411, 0.5180654685726762, -3.321816654818329, 1.4059191120346741, 1.5829194384821235, 1.1614485571380326, 0.5729174041320452, 1.2589042259712258, 0.9613307819791057, 1.6805346637235958, 0.8889671679857462, 1.3685687073702795, 1.5228916634627239, 0.9593978404036918, -3.5720990071743075, 0.5764703808990451, 1.9570481222929417, 1.1205479376087233, 1.5847613074232172, 1.6893238683922838, 2.3082557452171604, 0.8663764947520919, 1.1267908573957097, -0.3751095056116929, 0.42987459697353825, 1.3958694754989787, 0.7507026823483242, 0.9254508770072954, 1.2375939072509357, -0.56972902056028, -0.2883300951310862, 1.6796181462786957, -1.0792957691320728, 1.299267707484019, 1.558636191568363, 1.1834306186816268, 1.186824588143077, -2.4542365353218796, 0.03887205076578812, 0.44583501642810386, 0.6915365165176853, 1.470746180824433, 1.3416469047576727, 0.9849087348772324, 0.964688112749854, -1.4379714636551053, 0.7932547219966332, 1.3126255367925763, -3.5351093632128956, -0.029860666851543732, -8.3600068086208, 0.3568721103649579, -25.248332131954996, 0.4594562959606575, 1.6332358498243202, 1.288303318529242, 0.6553268965583893, 0.3029699812366773, 0.11389035981990342, -0.9741814909291288, 0.8084522585454464, -1.0992218799628588, -0.6075638522084496, 1.8390975924178412, 1.8122459746351922, 3.0063218054945233, 1.335954940283401, 1.7912019578761742, 0.45754458543692056, -1.8488467430846633, 0.7561438575047412, 0.8891429229357496, 0.575894824682307, 1.5833684363503566, -0.2334860026123197, 0.11062074560860266, 0.9163674240075176, 0.4165424248950381, 2.017169393352161, 1.4502508835891155, 1.5872856677334126, 0.6407667181468703, 1.381244016327938, 1.4280995794215399, 0.24398043236596445, 0.3750823620203093, 1.4254108202497173, 1.8768483192063956, 0.6038620372112887, 0.8677473696749779, 1.730015698709387, 2.3527499867110255, -3.1082717948371577, 0.2300575843489648, 0.40591721111984114, 0.20331336951150256, 0.4092755842899006, 0.3536096274290574, 0.46224766696828923, 1.5376712688493437, 1.6286404389136369, 1.566800845257169, 0.24302684649109005, 1.3513095121289938, 1.4662780793139019, 4.618286169745532, 1.4545747856499351, 1.8585943546158628, 1.1300056425598737, 1.3277651898130036, -0.2700884182670723, 1.136052263693615, -2.5221548968424896, 0.5417098337913978, 0.300623013852289, 0.5995523822011029, 0.801523511047049, 2.3965437413503614, 0.09334658115263217, 0.28375389865009937, 0.23882012647789364, 0.5611361451479091, 1.4797317052974817, 0.8782267666381444, 1.007367872250126, 0.7985222708889862, -7.0015444849324915, 1.501151735948288, 1.445752282405525, 0.4527288109120935, 0.5274213029609023, 1.7194399586921942, 1.3555585414896612, 0.873632122708154, 1.2550559862828226, 1.15506862937187, 0.14741215920613712, 1.2342967868937567, 1.2049856646827688, 0.840901478320312, 1.1101965439429606, 0.025513138750253045, -0.06935756111337535, 0.38335803878399044, -0.2749596765945952, 1.0071284736866823, 1.3192655350834332, 1.0970485261742442, -1.6014921076343587, 1.5285446849779645, 1.152335724042606, 0.5281223950993637, 1.8662832767933575, 0.33931841428515963, 1.1384545909720212, 1.6543825372718666, 1.8776042877776977, -1.138124177906658, 3.658769285202727, -1.7527982773210051, 2.4883970817876806, 0.05382181830183198, 0.8827247946528022, 0.2714958886362211, 0.17479085179764042, 0.3501906660391846, 1.1345751938826008, 0.3172762927932514, 1.2597973063291066, 0.5114181309792903, 0.16462090099714996, -0.4402232652809076, 1.8760778845015076, 0.8267785160249136, 0.1575322021265831, 0.708673931798563, 0.6779585897947839, 0.6596968855613549, 0.7053305171698855, 0.10710641868755419, -3.6917836218211146, -1.2124797542252914, 0.64897281960182, -0.32045180722027855, 1.215723405833608, 1.6217928610437857, 1.3854946674054287, 0.705532811078853, 0.5294801026676527, -0.4059292858372814, 1.0714777313297892, 1.3661517514250692, 1.3661517514250692, 0.9694930594299879, 1.678460557988226, 1.402553368339071, 0.9965794417564305, 4.189618175107865, -2.5097158163948214, -3.054174453202937, 2.5239522619672035, -0.7500061692469402, 0.7209181697415721, 0.8377654057831476, 1.207473976617562, 0.9885178071670154, 0.8196055561163411, 0.23282370693804053, 0.3926804948060355, 1.771512367226203, 1.7144991224385617, 0.33461580245136613, 1.364071019863995, 0.7821439335913458, -1.0984495142036041, 0.34498497436371406, 0.5130646835835301, 1.353323162388438, 0.2537287416179537, 0.825561742157494, 0.47714875483245944, 1.343371875226072, 1.2450042233860652, 1.7545078244482335, 0.26365926683863194, 0.8644479143575065, 0.5605023446844, -0.6094501407270377, 0.28525308526152293, 2.050393565500569, 2.5849817452679664, 2.3664466852738064, 1.1488035255258395, 1.2188236471534974, 1.2640964668140464, 0.640440593050272, 2.012836425225882, 0.5815499838525752, 1.6894908768678947, -4.311394839383924, 1.4433856581425062, 0.6984111796024751, -3.537840762866388, 0.7599282875492531, 1.3645189777050641, 1.7306850722471119, 0.9659042148178439, 1.7210998413694842, 1.6522673607980416, 0.7599282875492531, 0.7599282875492531, 0.7599282875492531, 1.2262336603104704, 0.5305694027576914, 1.6581236486200306, 0.8116797805005312, 1.2473709024015502, 4.091324269671476, 0.22331700186507125, 0.9223259594812284, 0.7599282875492531, 1.1355366566080747, 0.7599282875492531, 1.5179813485645273, -0.14768665832170932, 0.5836361304214239, 0.7660523005707904, 1.2140473187682839, -0.200556290891911, 0.05189330627833019, 1.3084821406804814, -28.66588645260362, -0.4990619816931918, 0.8878062047458865, 1.3996969705146955, 1.301492032420286, 0.7869995125859905, 1.1256105420060303, 0.4945449029649336, -0.37792683655014336, 6.247531191169797, 0.25075551583915034, -0.15452587903357487, 2.3560788088355453, -0.9233864720197055, 4.575467418014562, 0.16946072288554218, 0.02178408305032073, 0.49885134372719786, 1.6244241075032873, 6.671479955134798, 1.380752502203726, 1.149582731400437, 0.710547837154189, 1.3415791153013183, -0.6059174051500955, 0.25757748170244976, -2.642694436010485, 1.4335180208206473, 0.26838612643043913, 0.5485735382795327, 1.4882802230709071, 1.4641635304271743, 1.8384282203736442, 1.4154688471501506, -0.6096780321400139, -0.21316519028585965, 0.7259578990768669, 1.2859097308871394, 0.32209241969034685, 0.524328790744902, 1.169156348812803, 0.6287058917572216, 1.1055342760597784, 1.2824119665157696, 0.5781468108723179, 1.957838594759105, -0.24231341030640843, -0.49902103281681043, 0.4903781215535187, 0.6812467016001958, -1.3452044134000967, -0.1966345073566159, 0.7599282875492531, 0.5891964113102345, -3.0188801083603622, 0.9534398425999863, 1.2557705124488912, 0.9332768144805912, 1.3283133650252854, 0.4893719452243378, 0.956493177528429, 0.4606129843807317, -1.1046088924552155, 0.9452461755408746, 0.7036836374183587, 1.1280404881852701, 0.29336019433592886, -3.572290542945809, 1.3122886400527245, 0.09020450006145389, -0.17447787493593672, 0.1699896727321211, 0.1699896727321211, 1.1643911572276004, 0.15326554369237297, -0.11086056283630766, -2.716682201353265, 0.23086527191185863, 0.12571185989849684, -1.3262666146713347, -0.32141220585829866, 0.587383060435266, 0.8335473704518421, 0.24992608899025087, 0.1699896727321211, -0.17969189613196027, 1.2138433445102508, 1.781942238591864, 1.5496832472866042, 1.0294235672440721, 0.3841244647079223, 0.6616079437746916, 1.245493231466648, 0.9809637391589735, 0.7207374548597505, 0.70039460772681, 0.3723912188609829, 1.3163087938185778, 1.0269570207589043, 1.057387423294649, 0.9564325374266464, -0.4519476815782877, 1.4607551787675184, 0.9311950353428979, 1.7082504489440813, 1.4441021446558695, 1.7534471239362102, 14.404379716275004, 1.5513490863825374, 1.6295505483050756, 1.7752546285657185, 5.064407445435108, -12.361062495385234, 3.160473918609534, 0.39696651767516844, 0.25537498481671944, 0.019990533874768707, 0.6594598120391597, 0.6199889842962851, 0.6364590311502356, 0.7207468206282333, -0.7423105134703815, -0.25513252019402977, 0.8003018993572919, 1.0693972838378392, -0.11333985975023356, 0.3170402010665173, -0.3124276327441474, 0.6179829261143001, 0.722102190178206, 0.7599282875492531, 0.7599282875492531, 1.22668922764271, 1.0463857707871371, 1.6753926317935637, 0.4200679973516641, 0.5956855229282472, 0.8222823296719849, 0.7599282875492531, 1.880859655814623, 0.6715280599784625, 1.574847927716171, -0.10499239696857512, 0.38791781463937397, 1.336593013659388, -0.9468283674221086, -0.4266771352239137, 0.45327198310209693, 0.37279323348171434, 0.2226344830131689, 2.649173039274374, 0.9191749804288604, 1.2467198543985196, -0.49900026981274104, -1.2355282318026852, 1.5758156631335463, 0.9349442114383373, 1.0752531241272105, -3.1333335233422743, -0.8606556300262603, 1.8542058907786694, 0.7201034780039989, 0.7017009048272111, 0.9198205663834594, -6.8737985620560265, -0.5068115774403601, -1.335501266739281, -1.0768616280420253, -0.7676906669317949, 1.6509129233773545, -0.8518525158057845, -0.1833164764521075, 0.84267084088886, -0.006756719181824883, -5.4886437407914705, 0.6900432557476807, -0.5568071510556211, 2.0320592844530982, -1.4939971027463363, -0.1959893841768782, 0.5244498663903775, -1.386531464603331, -0.40006095990903534, -1.020264028593832, 0.5896732253638541, -2.7696809846908717, -0.5933220998249861, -1.6634982808023902, 1.4869293783159339, 0.3392320593052226, 1.1638821702661561, 1.5565403709591523, 0.9581209521123762, -0.00400687125043106, 0.36275286426770303, 0.43276393590145645, 1.0583076335603414, 0.8167266768267412, -4.175359515766902, 0.2159009988938588, 0.623326531200067, 1.186850758566328, -1.1142657192707799, 0.6801260901480671, 1.0941842137895839, 1.649565985282453, -0.06697730156219689, 0.2716703734243884, 1.0596842944870755, 1.6673011752738987, 1.4510109961047473, 1.7108708998910236, 0.8232668435519385, 1.1171365105506923, 0.8944171401291946, 1.1270664074052934, 1.324040300783987, 0.6498009146119735, 0.14075466635030967, 0.9754096764157257, 0.955346127208638, 1.083591910107674, 1.1342093282843044, 0.28646290238374156, 0.3124810267721282, 1.0537728240116995, 0.03902043511001965, 0.5739070245643206, -0.778265191773912, -2.6266106581582473, -13.825138261065023, 1.9323930768628201, 1.8028330310808272, 0.7237097286758803, 1.447796252059235, 0.7193802871425343, 1.1704128346318128, 1.2331984805958958, 1.1181468977583515, -0.06009000022051045, 0.5919015736088146, -2.0840019535603678, -0.7257377543848209, 1.3102159666276019, 0.9483909035486753, 0.8753788608168821, 0.8594960476342502, 1.7631033913514613, 0.883159239966468, 1.4942870332561822, 1.1520815932204187, 1.4356544464559469, 0.9745716535521498, 0.5334109867518255, -0.9615443283604316, 1.184958710189427, -0.018574177417578747, 0.6748424074251589, 0.5639833800802843, 1.540777054091359, 0.13052412499991287, -2.8685508556532042, 0.938225509807124, 0.6731490327262446, 1.2950808707889396, 1.2804901968735347, 0.8845715555207491, -1.0557590496745017, 0.9657454021526034, 0.6645574728410377, 1.0264562141775833, -4.038164319405673, 1.4657181796893861, 0.09214810495709222, 1.6852263649496462, 1.1672543057706524, 0.45188823292946545, 0.7846453242710382, 1.6400212379386063, 0.3749994679018812, 1.948554000567664, 0.7321021059409044, 0.8467693385860129, 1.3764006124712997, 1.5514104171494312, 1.3513553636282694, 0.7302046120269006, 0.01311433562011935, 1.0407678820657393, 0.7803906087694209, 0.5478725580539613, -3.181015977131027, -2.742515887636143, -6.950224140216708, -1.004274610705921, 0.3820182299682461, -2.125488528348246, 0.6232268258917573, -2.874609123417163, -6.096818557522499, -0.8512453339560616, -2.9821913176478145, 1.0353458045267017, -0.5659812525842037, 0.290338424614077, -0.347778234352921, 1.6096809587651333, 0.18686951557034231, -0.6073317671452249, 1.0713711867473374, 0.7417311204778451, 0.9669760520178091, 1.1826590830445707, 2.009500889687608, 0.5946237586852656, 0.13090024288476323, 1.2044087014319536, 0.06067371560169316, 0.5603830362304215, 0.15779328309566845, 0.3242997518974683, -0.9587774144532325, 1.808462956261052, 0.33922895487270466, -0.5251520812043553, 0.7854508168911369, -2.8763293849536162, -0.9742022367719908, -0.31791022888262754, 1.444998034113121, 1.2710568006115202, 0.5060280079816647, -0.6291005244444572, 0.37676438276214774, 1.1706366007958344, 1.2153337463767588, 0.23081777672961196, 1.3594471597945745, 0.8139448126914609, 1.154517466806346, 1.4349333207970572, 1.4424389235076214, 0.12890271199067888, -1.076251204918787, 0.9365420693385214, -0.6868775922203658, 1.556288333388162, 1.5580638179925554, 2.183641075517764, 1.492740166756933, -1.5217308391862, 1.7400733483241952, 1.2985756432070819, 1.2468746604162584, 1.866770302853975, 0.41884897194317305, -2.6954244912817424, -0.4606222884842671, 0.7916261068765318, 0.1378873066783111, -0.20553390428369248, 0.8347838707064404, -1.4113176859078995, 1.3989741402215732, 0.7640145003050772, 0.845452768637752, -0.5465380231564884, 0.8256853481899588, 0.8705226500479885, 1.8661097583787878, -0.1262247075815626, 0.6744901971755956, 1.698962592796488, 1.4925867035373452, 1.5191408518629015, 0.8606107539861406, 1.0998886210224668, -0.5630885204664593, -0.6556684986341209, -1.1194435501004862, -2.138665841977683, -0.5324489324872619, 0.8160006937425627, -4.813583245861499, -4.906399205837534, 0.5938902930963339, -1.9017910460500176, 0.34844519596894163, 0.13653420383764503, 1.0785805427377149, 1.0797730760708493, -0.7122043182468987, -0.04519786795348943, 0.9552553781078346, 0.06513883838197124, -0.28880863310895233, 1.0957946344530558, -1.81927890153052, -2.6985345810087744, 0.7855409666625989, 0.13653420383764503, -0.3717762881136917, 1.320408698576118, -0.5571106260831674, -1.4151758558477179, 0.7599282875492531, 1.0205788056016019, 1.0415811775954933, 1.0690540022505108, 1.458139400548415, 1.1681672798442324, 1.8158877655763324, 0.7599282875492531, 0.03966479267498471, -4.206635827394061, 1.4504382253218568, 0.6269936198672464, 0.34485948750227813, -0.541281655991652, 0.599156806490677, 1.2712990666874149, 0.7795783017710789, -0.5007441633390911, 1.1573021199222362, 1.611301575964353, 0.18405348918937992, 0.8452816722976795, 0.3097270855743964, -0.1830382054444232, 0.6398485081430348, 0.7599282875492531, 0.4036164704488451, -0.12121687195806455, 0.7599282875492531, 1.6486627354888732, 1.4177029141721609, 1.5249484021367885, 2.0455949394432746, 1.8131594089187408, 1.0523391920999177, 0.9079999698208926, -0.4386921404327634, -0.4059772152596649, 1.1824086325375374, 1.8680101948768058, -2.530831849825164, 1.728826560118555, -1.3477825210711807, -3.3612202877489845, 0.5002509121355613, 1.3144429629811059, 1.1989131416194763, -1.8902860941070994, -0.39410652800080315, -0.20674512937824743, 0.8042416070527515, 0.9474560481316151, 1.0465565464616307, 0.4105925498029633, 0.24090028021882764, 0.9201899098845748, 0.6196866812346834, 0.7872166573533985, -0.08163590691164815, 0.9256968004517612, 1.7672100209819361, 1.227318336983737, 0.9359600415024122, 1.346581928295784, 2.641361944025399, 0.9373774463457933, -0.038498946214387625, 0.4560418624868039, 0.9443207493794991, 0.2653115771414256, 0.3900574351288102, 1.243461217888076, 2.5018705854308956, -1.427990783373203, 0.057280887314713994, -0.48485010753329216, -13.199570488438653, 0.6389839058023425, 1.3905835326554434, 0.24482223592495544, 0.24861214463591413, 0.8674753574368782, 0.3847732115609162, 1.871570190929741, 1.354011761791673, 1.326812052468706, 1.871570190929741, 1.354011761791673, 1.3062004047872333, 0.7228173274110757, -6.992010703779259, 1.3188243265595898, 1.29917619939099, 1.5202462032544688, 0.2536702100949745, 1.1976027050344453, 1.4416536171256231, 0.9907181627411626, 1.3934510470994577, -0.1338783881642909, 0.7229275213208262, 0.33807227554444097, 0.2607923383909795, -0.8669421802526079, 0.7455350866163298, 1.7407327897094145, 0.9937898564219669, 1.858086883220475, -28.75768893917303, -7.830258220219535, 0.7998538630358737, 0.5485263723456837, 0.6311591121440475, -1.2211148580188966, 0.5925022301919383, -0.5354556564336651, 1.2108794897756112, 0.8276804536911958, 0.6530600976358004, -0.1428702055676123, 0.03366089934535918, 1.1199890488101796, 1.2699790533442445, 0.6375232500580481, -0.696332190104892, 0.44607775951516637, 1.4692524469510864, 0.9302908664193463, 1.0775497596299524, 0.7313716581411711, 1.0559181730691634, 0.8800046202622595, -0.42159442651246126, -12.041960964788448, 0.786586643881639, 1.0734982273422904, 1.2298841750947451, 1.4057937289500082, 1.3277912643410748, -1.0108587686905535, 1.354011761791673, 1.871570190929741, -9.08736657758307, 1.354011761791673, 0.38189253015008817, 0.09504067157207507, 0.44359396622030844, 0.9795179479350759, -1.0369361300839444, 1.0921148578333117, 1.4418817423772794, 0.9943322102881619, 1.679168687587776, 1.1780168672526563, 1.1780168672526563, -0.7706336831890971, 0.5125564704846775, 0.053170794221185064, -6.62730701548031, 0.19962579674288147, 14.251008153664948, 2.2525498647984437, 1.3027728571057908, 1.6163644927384297, 1.538561646757653, 0.64577284852857, 0.6438426407761068, 1.191201099416307, -0.07887837320346011, 0.9052035035874848, 0.7308114376813376, 0.4157532800015381, 0.7445137715043669, -0.44404397594401157, 0.5132354006805824, 1.6065158837403992, 0.9108696161072065, 5.166760353062157, 1.0975010905093225, -0.25717555940660664, 0.21372118511829372, 0.7195373211975752, 1.41684940540241, 1.3112586252445424, 1.3661875604668872, 1.3837698336013198, 1.8679968218943186, -1.953141694590244, 1.6657960180839124, 0.29996797408715337, 1.368385050809167, 0.08287935962258666, 0.5675143992055044, -4.813279945354836, 0.2841277478733931, 0.6907921923001737, -7.618943009308369, 0.5831912178649364, -0.004024279001777931, 0.7599282875492531, 1.5249484021367885, 1.444783940973352, -0.051022629971229555, -0.8172632799748735, -1.286393341128979, -0.47211056053378575, 0.48356885547638273, 1.8966046108276797, 1.3039379807497857, 0.24250863954169785, 0.7962764221606583, 0.07287528345444363, 1.4548309688970364, -0.4123500598112494, -0.18413706286242268, -0.6886658924729772, 0.7817070292625318, 1.554818306947609, 1.3673938074607637, 0.8963452730002046, 0.6783614640656336, 0.2662462868676665, -0.2700702215226807, 0.9055788227541769, -0.3217539006416026, -1.2820182406183784, 0.3380537618626397, -0.5747694830483476, -0.09042582891048376, -0.8451873248735439, 0.5156636815930831, -0.04545699272590223, 0.9873907558014963, -1.0909949135982706, 1.566242562848615, 0.7825603906756793, -0.5867957753387045, 1.4210547979547048, 0.10196259744103744, 0.14192780002011418, 1.139840330415592, 0.5024206332757832, 1.2279029767467355, 0.00047663523900745575, 0.29878716326567595, 0.3405815282290815, 1.1151145687406185, -1.3753081239258973, -1.7601815566797607, 0.31749294695298297, 0.6137484012226346, 0.9685055694945351, -1.8822264122968568, 13.313037865972204, 1.0145042744129444, 0.8534662712508919, 0.989379231196834, -6.961756831420731, 0.08618258933924974, 1.1893693427815981, -0.10228575625043121, -0.3544479420852076, -0.7396161402635741, 1.3631141107448763, 0.32924519441440925, -0.34592851091996624, 0.29453311177095054, -0.5537460648471112, 0.05050655706199356, -2.874808926176163, -0.5334310533985199, -1.57004433821344, -1.7512242986849096, -0.1363374469997292, 0.17828752194347658, 1.2630078122993635, -2.5210709420278232, -0.3092744012631846, 0.9433812520905577, 0.6729181286355859, 0.029993103087852607, 0.7847561607913977, 0.880412070915361, 2.070437917422964, 1.948526918558475, 2.1731527450126857, 1.6530218076297205, 0.8663054992449524, 1.8179170764322323, 1.7080256491903627, 1.4258004783731817, 1.4489016350293242, 2.007407261594959, 0.6324031538723279, 1.3656598698378954, 1.149221156694315, 0.6578899869272541, 1.8547268596926403, 0.9796392582833026, -2.2097660432851467, 0.1862570099465933, 1.158785482687236, 0.7207468206282333, -0.8852505228287618, 0.08730950860763181, 0.8432558525152744, 0.9017588057508575, 0.4337865032602666, 0.4171817868950591, 0.18593408087275165, 1.3606406620245675, 1.3311106711657528, -0.10615753058003177, 0.9305899185898034, 0.8906435219352457, 2.605886704427404, 0.7005974481865779, -2.520990704585851, -0.6768974693574518, -7.385454561869506, -0.11272994350524781, 0.5403532296945217, 0.2650780711330675, -0.6985423410601899, 1.0821877568769318, 0.9165862900073608, 1.0402683229183394, 0.8520836686123092, 0.04548738481591122, 0.3243113858034089, 0.645573692221957, -1.3636551993209867, -2.4168588287605237, -3.820759886066132, 0.6825129248715179, 2.3194310597222163, 0.06465889531153077, -1.61476123967563, 0.4034221397629494, 1.7222677327268754, 1.1310827660346345, -0.03325990146570515, 0.7821094033926385, -0.9967506801445211, 1.2000248614615805, -0.8026102538178891, 0.9743618016455714, 1.7648166406958024, 0.6811667015301237, 0.0629034226774138, 0.6731901589577707, -0.655673088381772, 0.7288354701276826, -0.3501077555183577, -0.08633589392341631, -0.21733150516442526, -0.2749762007765578, 0.9380907463951544, 0.33183846081259744, -0.016531213025460234, -0.14378699776499795, 0.2606327065034182, 0.4414743833568968, 0.8379260544783139, 0.63307300964361, -0.1441060370521928, -5.066761381524893, -0.3798346744556922, 0.5740713631440322, 0.34556025610371965, 0.8559700968072311, 0.847372283874041, 1.0036318559529072, 1.4385512128695643, 0.052002628931903716, 0.392306070301494, 1.5456809877824023, 0.6197768084267684, -2.2306821739222666, -0.9469075662837162, 1.2769853893999963, 1.4367872487285218, 0.8961053727196328, 0.43131436629263487, 0.01550128176560664, -0.20613568542479113, -0.3306086734445569, 0.7360996605322947, -7.688155468183367, -4.972626470122826, -0.8733396260763905, -0.14286548471238694, 0.8753770586222733, -1.2634383093231916, 0.27444752491349916, 1.5965766788562394, 1.1288614342420984, 0.9774251328973979, -2.3845783794851756, 0.6951131077670416, 0.7697277593323412, 0.8698881716950352, -1.831071237056879, 0.8564225822337395, -0.14759295762133515, 2.1590030534876807, 0.9114254187603478, 0.12302075533068088, 1.3175391933874196, 0.8764596870233311, 0.7017091359666378, 0.972919828623508, 1.5012399642477987, -0.21809656227458282, -0.214592863759879, 0.2960352059424343, 0.5605388206578344, -0.5856334914219488, -0.010694120929958528, -0.37102527202647384, -0.2434344669206047, 0.7155748609581651, 0.6958020639744865, 0.4941682056632699, 0.7136154482273059, 0.47929300048291945, -0.16958626324427548, -0.15205063807135324, 0.3232888670151586, 0.4980114942417059, -0.7580405378712839, -2.089592737196143, 0.7960752087296165, -0.19770670816307365, 0.5103724082183194, 0.4442422884140642, 0.6319046579956895, 0.2865594288984237, 0.5515151588989439, 0.6318271810388754, 0.9867412497676257, -0.049761085550526024, 0.24994688967631615, 1.0453845290102324, 1.8577721732527845, 0.6074039729298195, -0.3908671170503125, 0.7975721773583638, 1.0076532119383403, 1.8078593382241757, -0.3819968082509275, 0.944417379842339, 2.052825422790085, 1.2066120671378298, 2.48041943070592, 1.4387010464976555, 1.5635674763100431, 1.346321581248647, 1.1247831797442023, -0.10848942659762609, 1.127460327292433, 1.8656517294493273, 1.5697698701122906, 0.8825284127998586, 1.842717932243684, 1.0662591585612313, 2.6388217240442646, 0.8226032444596045, 1.5557769428386687, 1.3322912696866869, 1.6982352618886185, 2.0895621799612654, 1.818893843933842, 1.038954900950218, 1.4540495886115143, -3.085768987576991, 3.378634369889712, -0.01647624247183585, -3.1256189232156855, 0.15976367805438887, 1.0492604948865973, 3.865300013892232, 0.5251787656070613, -4.91101441616973, 1.4872169543266793, 1.4088509329643824, 1.6091930219999737, 1.3650039668036684, 0.9978481025211403, 0.9109249416461302, 1.1404691854220892, -1.4184260487860212, 1.670158009401319, 1.5561091223363426, -0.8059486445219578, 0.7211900004162636, 0.5434913618340432, 0.9090890186616539, 1.0857174218091745, 0.5631095708907251, -0.27846353470544905, -0.027971454542767754, 0.4945507958708928, 0.5541917502080107, 0.7478944694761961, 1.1228375422833656, 0.588990507384152, 1.074735810679754, 0.2839218279422174, -0.0727490531709977, 1.0593105099049291, -0.03171502451064381, 5.094386421541996, -0.3740697604224219, 0.7166516614028444, 1.0141238950107239, 0.8395383519336566, 1.4884463966775148, -0.26974686729280706, 0.32943954587131064, 0.9628432658241541, 0.19785124322395561, -0.8763922200190274, 1.1781187373391766, 0.8800928829133118, 0.679304775785831, 0.9711712091477677, 3.437373508575786, 1.5956925049685105, 1.5951651485878922, 1.362524547727774, 0.9218511692736658, 1.4011744170932843, -2.0196118752010976, -0.06662405003537078, 1.4241618635492959, -1.1054884868508752, -0.17727245494650626, 2.6219028630456025, 0.8538498668643633, 1.7412986514372246, 0.8349063825834931, -0.12006742229222225, 0.3153990915717979, -1.119232376273907, 1.0788690363094788, 0.8096444777702572, 0.6785117120456033, 1.0799562142121704, -0.16706012519975247, -0.7927267797394532, 1.3442294708184306, 0.7494453166848383, 0.7603187847267714, 1.0498531426717175, 0.6311010782701119, 0.49809861177294246, -0.2300230085675357, -0.1929468967608913, 0.5413880204345667, 0.9397748477495814, 0.5861893659489587, -0.23643373533864745, 1.2051834728960433, 0.71098160564406, 1.3052416191599183, -1.3925299696120508, 0.535817512197708, -1.292413161870988, 1.1979070584065852, -0.7572987907281715, 0.9596031111973905, 1.3661517514250692, -2.0526748699133046, 0.9965794417564305, -3.117195430046643, 1.325264722528715, 0.877232327085689, -0.13501961479777283, 0.590070977256673, 1.056817940265006, 1.3485798097273045, 1.0267042040087817, -0.7101722305199321, 0.7087499619722485, 0.6812417577459555, -15.784539564498957, 1.4933670480854535, -0.08165944689063051, -3.2776722887717415, -0.9615355351828424, -0.8044507852053026, 0.6886962829900857, 0.8008388277122722, 0.27549058679939553, -0.2211730358323705, 0.7291606757100678, 1.2737379460701774, 0.6544657624450876, 0.21506321601325193, 0.19083205725640315, 1.4796063107034878, 0.6107252866943913, 1.265559789341391, 1.125184047462284, -0.07597592025256888, 0.9686525498304008, 0.5962645684098982, 0.9522503384376075, 1.5142909025694256, -1.0665216986871868, 0.6844257038335776, 0.21841787976628887, -0.0709876717778504, 0.7428265000221154, -0.49460335798711086, 1.3117036755145182, 1.1764670654710294, 0.9876378567666695, 0.23161234198909147, -0.06653790383939659, 0.6095448539497802, 0.2891085334157979, -8.392246830495681, 0.7682954626408528, -6.514259007241921, -0.04638456672473876, 0.3417431420829342, -3.1824879607065104, -1.6028525099925286, 0.5455283899128776, 0.8362795106644882, 0.6096416493439001, 0.45099912248653695, 1.8676202791716972, 0.4712482402523175, 1.5400316765327056, 1.370504383286484, -1.679726782847319, -0.007954843597013622, 1.5670831947362125, 1.7054548996549261, 0.7552368392310019, 0.14724095925441524, 0.5884294907446396, 0.33233902186230846, -1.58708915074236, 0.03610759264088212, 3.9857211855558727, 2.93150528226092, 0.029374414421513867, 0.18934946618651613, 1.558557641318044, 0.3396711196638036, 0.8534983807676145, 1.6404306137109088, 1.314905958723577, 0.5297228176463715, 1.185192742589267, -1.1424300575028312, -6.185930215859648, 0.055632894931555904, -1.0223795293228173, -1.6851702212278117, 0.9752744423197924, 0.757689005961368, -0.685418195758999, -2.1109855131381323, -0.6289425665839874, -0.9435347039866665, -0.3325965960294067, 0.0381030966630784, 1.368249473321347, 0.27611556047151165, 0.9849705776608465, 0.24635543781640215, 1.5145952258041584, 1.236721914301491, 6.0289990625209215, -0.14027919881338227, -3.3148587503523745, -0.08999191528740288, -0.6430445528448128, -1.9615661808951497, -4.891412896259305, 1.3605765810859307, 0.2816950988352569, 0.9083863697375049, 0.5816500216348918, 1.3372837896040475, 0.9674956280745166, 1.1403612402459038, 0.5778027328531251, -0.2696919357724855, 0.3387424927424629, 0.20244691238157977, 1.1534365321320827, 1.2762737256355006, -1.4068311209258402, -0.3469209337114636, 1.277866595235764, 1.9728843873898645, 2.1563783644326264, 0.1545028124640008, -5.275764644506527, 0.7329558429573071, 1.5458604299019916, -1.0827485875527552, 1.3004282320398801, 0.547076237678319, 0.9959479942664043, 0.908380465382047, 1.1395236461813265, -0.5996332953184714, 0.7513741024794643, -0.5259136027019865, 2.411013269959614, 0.7958130164882269, 1.3896824289495389, 0.9508668288051724, 0.4588436094963112, 1.519201989026673, 1.644269661242705, 1.5506369006246885, 0.9745909879163773, 0.4792373822332292, 1.7705255541217417, 0.26566259347317395, -0.6655853196292093, 1.45855216169455, 0.9790637694021405, 1.9185958031526176, -0.5248553302562072, -1.2291691511444909, 1.3054306108122267, 1.0269102948226894, 0.31213240243683604, 1.440315731933721, 1.3874001053042364, -0.0469053677878227, 0.7669322599206082, 1.2874800320037176, 0.8476558267007093, 0.6692726378366626, 2.0678925521522826, 0.9523166926999214, 2.2873002719495954, 1.722082398798883, 1.3407147552890584, 1.0915069766879673, -0.02051490629108263, 0.2788781971272746, -33.09036830623287, 1.0402662203274244, 0.9812052151181806, 1.272808115799059, 1.5168950672421455, -0.08807726848865335, -0.2304842226723022, -0.5429624554646258, 0.1446695467205254, 0.024404480412666046, 1.3837931834054946, 0.9070994819838267, -0.2355970105591669, 0.9620390258287065, 1.9837554293265998, 1.2512494657352946, 1.4577960974034603, 0.6065282903672633, 1.1024666969597996, -6.663398692327877, -2.025837625072526, 0.5188864978234606, 1.3142838733329154, 3.490404430460616, -0.6715594991154205, -0.6073028719670389, 0.7862840135547313, -0.1831845852909296, 0.5546630093000592, 0.661058560265384, 0.6550491626793671, 0.929108687165849, 0.939124050349526, 0.5546630093000592, 0.5546630093000592, -0.10676625448868302, 0.587715661109423, 0.5425137364774598, -17.2082409017185, 1.2569293999876046, 0.6691893593410608, 1.0342362312868636, 0.9795752103902639, 1.388425669247025, 1.6079202559299002, 1.084281670741889, 1.364070049790303, -3.7531751464048217, 0.8952472155678682, 1.3230924703936202, 0.4316486730299379, 0.2143556363563404, 1.0078305025885264, 1.3211692891556714, 1.6419948882153252, 1.1738209752926354, 0.06663616480704329, -2.580951533402576, 2.0500428918732636, 1.6739164167789011, 1.0168431360729682, -3.87950758157948, 1.079948903529098, 0.6310413470297218, 0.2702641791799931, -0.3679889301971805, 1.7667508057323553, 0.6568703948581635, 0.4658080357019496, 1.8694028090013668, 0.6345713557864519, 0.2511054887470126, 0.38714507141965926, 0.6737243665609945, 1.3269799600386338, 1.570951086197407, -2.6097290373826065, -0.6662279557835598, -0.27612269723512284, 1.0224454360330733, 0.8324556809940777, 0.19897154773678913, 0.49453032403835273, 0.6641466275462948, 1.8917636929171602, 0.04239127468821949, -0.22144256132733628, 0.5546630093000592, 0.8976930530898128, -0.840028946401337, 0.5302258069646762, 0.6268365850675419, 1.471205719776848, -0.22850657266995414, 0.09659345328945537, 1.0024106484245798, 1.424269967080726, 0.8281045555499698, 1.0024106484245798, 1.1591028421756513, 1.5971522347892673, 0.7669322599206082, 0.7943158800201443, 1.1218627550293954, 1.1400450203827628, 0.992611126461732, 2.5771316075165713, 2.1409080745444675, -0.49163690407385774, 1.709322756188856, 1.7858922042568655, 0.20895410098772108, 0.13454477784712168, -0.4995522155114439, -0.8260944934775661, 1.6189200926725216, 1.5489119101632614, 1.5069170936966592, 1.9478902495164399, 2.038273553248818, 2.7657282917613437, -0.8779847351580404, 1.3335507746333366, 0.014182162934551379, 0.7996212967626491, -0.3988609792265082, 0.598392528492587, 1.3661911974036083, 2.1829851387314205, 2.1731574216554734, -1.7599343012946542, -0.06183211095448485, 1.8925235984357756, 1.4670937934197297, 0.10112630522935831, 2.2208821906499865, 1.5114597839264725, 0.19016774292013838, 1.7801618217623882, 1.202260866285113, 1.2054616594855123, 1.2314185936575162, 1.358915839873237, 0.1422321631423572, -10.71994749887644, 2.103622389817382, 1.2591969134904875, 0.5546630093000592, 0.5546630093000592, -2.4970459402557648, 0.012922588191898954, 1.5905955063681927, 0.7898884434541992, 1.0057854563044493, 1.4607597349158497, 1.482484638735615, -0.3300972076594505, 1.4479433609536676, 0.8504599099814619, 2.2291242303142944, 1.6105182540451473, 1.6016893460902464, 1.4957572891058242, -0.9824115445825631, 1.5357390167415816, 0.39938058712224167, 1.692826179869817, 0.4392078998479211, 1.6551106777520717, 1.1545038024782681, -3.996567031509369, 1.3903436952984554, -0.020158240686423107, -2.0771298807875387, -0.2839957655186133, -0.10721791644685227, 0.16699690463782435, -0.5757235095923338, 1.1138247973315303, -0.05462996478601065, -1.7148742298423945, 0.6217189465198606, 0.6150546036050238, 0.7989046022295307, 1.3374439811723287, -1.741483868395485, 1.538676179246977, 1.307370403756447, 0.18818324888427415, 1.4437215870488684, 0.5905380238026219, 0.6403228602530273, 0.8401651823019275, 0.5830187722583999, 1.4000337300179824, 0.8668260961359286, 1.1344561723867392, 1.512878246748834, 1.6448631354820276, 0.7869595630850871, 1.296992769025278, 0.28448605125082976, 2.01884343448124, -0.8713696754039201, 1.3283211093901166, 1.3401515804712794, -1.3288514563179248, 0.8355458408263529, 1.313035430548693, 0.6734649965464525, -0.021550784208123758, 0.24895157348981742, -0.39908975607326236, 0.07614023756598473, 1.0976348841023655, 1.1311605026323293, -0.41568731804684855, -0.44605127911691855, -0.03635401028619781, 1.3225562153353123, 1.2576808833473525, 1.3962092193030124, -0.24499712276833754, 0.8340755125098238, 0.5546630093000592, 0.5546630093000592, 0.5546630093000592, 0.5546630093000592, 0.5546630093000592, -5.043259766940511, 1.0588555893845373, -1.3173666257730783, -0.8016096391823392, 0.5195696270181133, 0.2878540480247643, 1.3112563862388218, -3.4494867493911316, 0.49032079361135406, 0.13213172211383697, 1.6387476900931972, 1.656802730038507, -0.8031496858793373, 1.8342556711392244, 0.4974680378701726, -0.8604296341599632, -0.08894339020036154, -0.979252724789193, 0.6312478280739288, 0.0956309757726505, 0.7396547953760338, 0.26225809464199135, 0.20890986219078345, -1.3420964211571385, 1.2202768986836303, 1.5126400726589269, 1.4039254604550329, -0.12550125968980233, 1.7127873308072055, 1.168694832592498, -0.022229898572468237, 0.8654101831999167, 1.4045209386161186, -0.2191739307750403, 0.47894118296520083, 1.2860202579528837, 0.9164353097668253, 1.6436567501307573, -4.204845670939902, 0.548543009141435, 0.47368535052155647, 1.5406716369333864, 0.33265194026555894, 0.17398346475634155, 0.735636429121037, 0.6483588368903143, 0.9064701429838719, 0.6540020701834817, 1.5919795842144207, 1.4302376272434414, -0.9150037478231158, -5.136194922050431, 0.8862039231672451, 0.0025546972774013743, 0.9774463805956959, 0.7423897005993276, 1.8986539533338465, 1.804062293829052, 1.1414260350515335, 0.7002099286810208, -0.21802970954737236, 1.9811770945222573, 2.5807388097644743, 1.3799455873275408, 2.4898036548489273, 1.4306605879467844, 0.38094702633008465, 1.5842500257005006, 0.1209030280996683, 1.3337306425518232, 2.4231153242157526, 0.6977977898138219, 2.0880106910963736, -3.1442433665655947, 0.01275459625255046, 0.8919706387780162, 0.8919706387780162, 2.556108665043986, 1.4832838572401548, 0.8919706387780162, -8.474361310839411, 0.8919706387780162, 1.3889614887193655, 2.0125683949444215, 0.4499199871124897, 1.1414793523127553, -3.3738094838475394, 1.7701481004086381, 0.5799636110968923, 0.17688056718960019, 0.9038390714833208, 0.6264482957059535, -0.54686915811877, 1.518011283307229, -0.09037619263874844, 0.8564356576891732, 0.5612989130345943, -0.24242692319736261, -0.25589266824024426, -0.7744618268033892, 0.5546630093000592, 0.5546630093000592, 1.3715566219071724, -0.4909262184194737, 2.0493518476887065, -1.538096263668426, 0.25715042017840056, -3.2045339857476995, 1.1216543281504077, 1.095021393418493, 1.6195790262391387, -1.13058926851733, 1.5894899065153634, 1.286628083583256, 1.3834663878522677, 0.5546630093000592, 1.1747782528898851, 1.6593551496548826, 1.2152286377771575, 1.0860945777026592, 0.17197346067843575, 0.3699363518552261, -0.6998558868819318, 1.342724825522845, 0.7308207044213656, 0.5546630093000592, 0.11778335991019537, 0.5546630093000592, 0.6388627103699575, 0.5546630093000592, -1.8574443537729433, 0.9369249422324468, 0.9496839313983162, 0.9204682198740631, -0.4457370973177055, -0.509739017789955, -0.05674602023738831, -1.0508504031332786, -0.1765325142098735, -1.2305362649013334, -5.3457874770669, 0.68227320113771, 1.813457943572952, 0.3517036200323026, 0.4101523369248523, -0.228128607637842, 0.7267516960746837, -0.8093742691966191, 1.0397526630721192, 0.9159208891978247, 0.10536611514376834, 1.6984002495859174, 2.670865649998754, 0.5472331449425489, 2.6449824197628784, 1.9764810637655434, 1.9723319750493564, 1.0594390471457396, 0.35079746685637114, -0.6437309027481796, 1.8370448694254127, 0.429845454448717, 1.770060105118898, 1.900001088227931, 1.5788494877374903, 1.7801756856667048, 0.8224854547185237, 0.8039081661268622, 1.9176368337785632, 1.7296569582989145, 0.5197935454155832, 2.0618767317687694, 1.5829810927157586, 0.7036146524147228, 2.2571253056086538, 1.3282581669683773, 1.7868420701340368, 2.1535834199891815, 1.7086436604713788, 1.5658165473347516, 1.1825061850229852, 0.7103122122651163, -0.21414178485972282, -0.3179106009407159, 0.8099304417640216, 1.2331942130898998, -0.15117645452627906, 2.180343956204829, -0.04385266239819233, 1.2324424073178206, 0.2115400432906521, -0.744177961965717, 0.4032010012793395, 1.8129549849389375, 0.8895772650129019, 1.5325470302962476, -0.01872892029142495, 1.415921081652255, 1.3718888848584914, 0.6671411748321675, 2.038273553248818, 1.251637702044703, 0.6796237095746794, 1.9582981257316392, 0.7034630005537674, 0.8099304417640216, 0.8921750837543333, -0.5640701506630396, -0.08707559978230431, 1.6423514908002697, 2.030030157269158, 1.656429728635327, 1.390237840647042, 1.8232011058990674, 1.6427074792023177, 1.8493526259815345, 1.165656677881741, 1.7435387129205242, 1.9592722520116967, 1.172845578312305, 1.077719829226975, 0.878801769842091, 1.6450575609417084, 2.776465165904865, 0.35441462224810816, 1.4673580094432306, -2.6781466563746283, -0.10127151074539989, 1.3413665234343826, 0.06168283211838296, 2.74224284914138, -3.1011547015384755, 2.015510296858265, 1.008166980231009, 0.6902410705380203, 0.7337211980084356, 1.3810758141956871, 1.7648006091248332, 1.436937718342343, -0.4368587714519191, 1.934257919867855, 2.203698846431806, 1.3831288494388423, 1.917423701488694, 1.597017743645137, 0.5019767118463749, 1.8218352506740987, 0.9151603762113512, -3.2457898198838753, 1.4185298872442482, -0.04373919630176362, 1.6060805472234692, 2.2133725016135477, 0.9674171074925231, -0.2329434924721924, -0.5209195768959417, 1.5949613662169575, 0.7177228430566652, 1.6045915672918323, 0.9521528964342106, 1.3603513451002578, 1.8567100706642057, -0.48851440320041684, 2.3940049899238276, -0.2398728669865339, -0.4403032087035863, 9.786440187848656, -0.47837722709315667, 0.64522706378039, 0.003593088366277669, -0.7957551299858776, 1.270913025671349, 2.5852279881420297, 1.9975586901260607, 1.233740214878126, -3.285428677133425, 2.008423674473728, 1.8119605703029256, 1.229475661104753, 0.22327557910406506, 1.009547581020311, 1.6912270993735123, 0.4222457465898991, 2.0880147223175123, -0.8108137573625204, 0.22712534143083202, 0.4032548108282229, 0.3655722835090123, 1.0108355439690762, -0.7076644387841554, 0.6699668706093196, 0.7685999945169145, 1.4194546271847726, 1.3599189282754802, 1.5742897596566177, 1.472838807602905, -0.6848796768836121, -2.7811043195203027, 0.23008221380545063, 1.1265626350725235, 0.7990043806424382, -1.2804823167892174, 0.8756264672762453, 0.9342747316180248, 1.4812160904150409, -0.19097250797793172, 0.6238999050210029, -1.1135035054076259, 1.2336022776853444, 1.3250293718721735, 0.23597309239067185, 1.3250293718721735, 0.7465423076095282, 1.4594972043493595, 0.024065851569339797, 1.69734752364173, 2.0254938494263137, 1.4791562474432005, 1.6856819748534364, 0.0018924363285020285, 0.2489652818014547, -0.3328666940752779, -0.3879736848062528, 1.7648006091248332, 0.24251465374602285, 1.8908855411936236, 0.4141430767897172, 1.3536257911007983, 0.010644348352496474, 0.7341591235283507, -0.08419195214505301, 1.2272713102656976, 0.1716985051358438, -0.8433116576593532, 1.4832838572401548, -19.763773402453236, 1.4832838572401548, -1.113536524427342, -0.9974245435145378, -2.9179874022425967, -0.7749917662941868, -0.9172251584390324, 0.19483532097464554, -0.03861749560083588, -4.460748076516926, 1.2602582656585677, 0.3556241832799072, 1.2383230852132683, 0.14611960254676168, 0.35851831725576033, 0.19411816466992377, -1.1205337632491361, 0.6289791474847332, 1.0306171912201918, -1.7659964717408005, 1.8534689712067784, 1.0889292739060905, 1.2281259429788582, 0.8740400622286705, 0.7852010658163875, 1.8744165260541577, 1.2649874737268674, 0.9149713264339246, 1.4107037109136873, -15.91002393408798, 0.8442759779668814, 1.3947767094715113, 1.3347864919333685, 1.1711253793075427, 0.1498137003058527, -0.013949527398600836, 0.08917360165377639, 1.436937718342343, -0.3638760373610228, 0.8187397228323284, 0.9396922522687586, 0.3254922111032095, 0.7581329746955089, -0.42606176345736413, 1.8977569270501977, 0.7095699243598466, 0.16777774361174486, 1.049318073688063, 0.6201017972880463, -1.4615851256331915, 1.4732346554582743, 1.6590552680601383, -0.7929058523561692, 1.5740273066237076, 1.5744008148270068, -0.22609603904524603, 1.4504437465622122, -0.5804726188127391, 0.12109027992928775, 1.019354382878477, 1.0050355118118435, -0.3991554335444625, 1.6298619042661986, 0.6333061047025589, -0.4580107004994626, 0.15901200641062047, 0.5546630093000592, 0.5546630093000592, 0.3640576975263634, 0.9443748273013044, 0.5546630093000592, 0.6460926525994506, -0.42990508994172927, -0.6780159918013169, -2.1406639724604295, 0.02043321245362731, -0.1805366743220956, -0.24429310762193504, 0.4822427991402988, 1.0813783477485386, 1.7624018905947803, 1.6145954924409873, 1.3800916975005748, 1.2947518392260986, -0.36492201045020634, -2.40387512809855, 0.2309987131741105, 1.3555967045733976, -0.21900833656879767, 1.3503470656022665, 1.2436916264187337, 0.47430150610104416, 1.432783308360195, 2.6075525254863035, 1.8958546153204543, 0.7904820004585023, 2.4086302841842535, 1.4556405079665715, 1.0917294597031093, 0.7467309219770023, -2.8861367233626627, -0.1997667689103624, 0.07020520217489348, 0.7590768821493717, -2.509289693501781, 2.753443100570649, -0.7301536186446818, -4.3180917754340955, 0.8542483532115405, 1.1209266769636999, -0.5941810022211697, -1.0247151699279438, -0.9203816957063434, 0.0692557810870865, 1.0131937465790641, 1.4806971168370453, 0.09710239945257795, 0.38974566151128354, -2.9528239501251416, 0.5773215461677101, 0.11209277338613828, 2.5874553859915, 1.1345934715864978, 1.3904763488952867, 0.6682858935240965, 0.21716459109980074, -0.7236599605912774, 1.5029196141035923, -1.0774147070808058, 0.3533941091454721, 0.3508197348976777, 0.9060069768646366, 0.892095226726214, -0.4893067502533295, -0.8016096391823392, 0.56015164060562, -0.10582159045439452, 1.0070934352113228, 1.4118869492709687, 0.7065798402267206, 0.24836730623515565, 0.3052116081338966, 0.06792536622067363, 1.1429535017180346, -0.1937670802850163, -1.5356961103428466, 1.895534162951094, 1.1755796131964238, 0.2363888782098034, 1.373865101708445, -0.39724762285551407, 0.9591652307287574, -0.029144734962599367, -2.725418680977856, 1.531949888893483, 1.948081455616704, 0.916336948078782, 1.8795206502103983, 1.5465372564970956, 2.2412563655861293, 0.38289653513032107, 1.8795206502103983, -0.3355484044983592, 2.0755634789516644, 0.48868190152094015, 1.5549868295899305, 1.7170729994598208, -1.4829557845758308, 0.48630517416661045, 0.47855366649610986, -0.8846997056775542, 0.45495127854285, -0.048265899064887624, -0.9084124726201864, -5.24240196558075, -1.4247278049891305, 1.0133701934116146, -1.1057000843927716, 1.8802028023441313, 0.7183004105421498, -0.7192046878782093, 0.40188792368521886, 0.12319250831932091, 0.008986405809087744, 0.5994855293749403, -0.06329025965281429, 0.06309116981739496, 0.057428733069759264, 0.3949347971283256, 0.2666434929648437, 0.4228238800251215, -0.2835516140922588, 0.4633442006586207, 0.3366978566618728, -0.10718822477787374, 0.01956653556594795, 0.2318744166460942, 1.1720764567988893, 0.11619414035224568, 1.038016339869149, 0.9400125424752487, 1.3377934269319551, 0.3073426936549609, -0.41330515501467835, -0.4380488129852588, -3.3426956751134993, -0.9603649886546805, 1.1510266286320792, 0.49591487519329525, -0.00211468386929603, 1.3712277477155326, 0.5992018548059388, 1.264326832768034, 1.36560566250364, 2.2402909518019345, 0.7690154887278334, 1.802430269479948, 2.1547960668414707, 1.6450575609417084, 1.9600665937847823, 1.6344203871009435, 1.7158341915339121, 2.1535834199891815, 1.0425795548870644, -1.600527645426756, 2.4257714058981485, 1.8756012443845684, 0.9802244592582433, 0.9514735733397884, 0.6774510645216832, 1.5907392921695749, 0.24669095284307338, 0.9467830674096793, -1.5502044540978384, -0.49241491875094534, 0.8403104653352512, 1.4107190523704296, 0.6158364046808287, -0.41751677724723246, 1.4493208164568567, 1.050249396628706, 0.4697847385434569, -0.8040118295916726, 0.685800239452287, 2.2208821906499865, -6.699908766764977, 0.6080131669681758, 0.4957819567446663, 0.7448966686237061, 0.8134147240385105, 1.013776074185748, 0.7570145933211215, 0.6913876688998796, 0.8617389463630158, 1.1867869462406484, 0.734332360357682, 0.4780404207564236, 0.8751983357136196, 1.3655725581317635, 0.667583265742152, -0.4799319522649371, 0.8649166384609702, 0.8524614775321698, -7.110234391812875, -0.1580189460377465, 0.6737477759553769, 1.013776074185748, 0.5056154705465123, 0.9969188602167822, 1.037797341117085, -1.0896818246637217, -0.21847512073508948, -0.70214854813797, 0.035019458873680465, 0.4811013162224125, -0.010359643736133989, -1.1100231387805934, -0.15917242896562697, 0.614282735393915, 1.9849356644464164, 1.8453236251876242, 2.1409080745444675, -1.6413021654168607, 0.12157001938474546, 1.9369092035189615, 1.8620005402571977, 1.488056863193782, 1.049929150778781, 1.900001088227931, 1.7055372859834388, 1.3296295197700057, 2.010255426084418, -4.6924544339094, 0.34849548327834967, -3.430854134522907, 0.6874000796726586, 0.5189004223158945, 1.1237930405994547, 1.6388906478153744, -0.44190997912142627, 1.605365673556806, 2.1342789111945026, 1.2833116572331587, 0.7874390172815239, 1.8478727883986166, 1.1965632177757177, 1.465985193731961, 0.8122701265139958, 1.0387740959815304, 0.23597309239067185, 2.5771316075165713, 2.1409080745444675, 0.4180140897072825, 1.4526102267424685, 2.5771316075165713, 2.1409080745444675, 0.3720228680208264, 0.8110315753591036, 1.459894826072356, -1.9151899928225977, 0.7727621441631427, -0.635421543926349, -1.7415762919985833, 1.6017280226891981, 2.974311224403015, 1.4991399990424803, 1.6335615966954458, 1.0573281477767658, 1.8778606966893119, -0.1302942344384442, 1.8343800800133343, 1.6604734120716707, 1.4480612153794892, 1.525602421756387, -0.603694531150672, 0.34478257247153576, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.29968900125496, 1.2737836314166788, -0.5900406418124998, 1.4458897609253463, -1.8609228090485368, 2.0874049025282986, 0.7315174657273876, 0.5425121418775638, 0.36627818636992193, 0.8862537768971105, 0.4578120487229542, 1.6590023265452305, -1.7430983979456336, 0.3086343634616846, 0.8345650082271159, 1.6761028026880302, 1.6615185478365022, 1.4715894153011846, 0.5193342184053394, 1.070925760069843, 0.4152793820412153, 1.6248634709849565, 1.4336697436514767, -0.19171740260699358, 0.8188138612810295, 1.7339677311198416, 0.5092312261999722, -1.1331385089259693, 1.411980476662694, 1.3333919771419658, 1.558709802731429, 0.4133680976570726, 0.9790927482331544, 1.6590816505544579, -1.050176186515994, 1.5111292763712971, 1.4623057466122515, -0.6399135949050399, 0.023290349118233994, 1.6698610215444918, -1.3147982239029221, 0.8488377398751935, 1.292607461150306, 0.7923125658325715, 1.69928220926172, 2.0696001672692192, 2.2244350834407123, 1.6806897132757017, 1.2749268737656592, 1.524557644052223, -0.36462909175513347, 1.904792914728619, 1.511421196731542, 0.3537945085825419, 1.0405069228997244, 1.2185639551410112, 2.0670304173661767, 0.7413591463961137, 1.3657959078619613, 1.255279385360766, 0.2501805562558883, 0.8209430437240507, 0.5838889030617913, 1.022312636513892, 2.0402360627618186, -0.830166176709653, 0.9503744122555592, 1.4044574276217814, 0.9930258159867534, 2.3703350512793597, -0.6659431143138107, 0.4618085652120869, 1.339176745163633, -0.6226564820927225, 1.447880913448751, 0.005279657396437015, 1.3220469124104546, 2.230048018062342, 1.4453429452140067, 1.3496998280835255, 1.6266339302394874, 2.1212045351745448, 0.23597309239067185, 0.590402777913016, 1.169372984981465, 1.7565406802472299, 0.7369840312611964, -0.22325374195801653, 1.1355999965604808, 1.5980414642934158, 1.63675511092709, 2.025794153206424, 1.7561688416504204, 0.760171984273739, 1.4191494797516622, 0.6095805117543462, 1.4344289039608884, 0.421440304822574, 0.7594732669360275, 1.7805503629467943, 1.1545038024782681, 0.547076237678319, 1.9813255327084363, 0.27302266128591757, 1.2119960706291037, 2.046188106781862, -0.5559239760520442, -0.21808197530633303, 0.6361399597576939, 0.9687611628878176, 1.5489107276924419, 0.04283885728815931, 0.9715519777722903, -2.0675931711106816, 1.1981090913657764, 1.594333427988685, 1.39616921225746, 1.71660945792197, 2.3703350512793597, 2.007131801937565, 1.5843849052755543, 1.6462915817022363, 1.6518144442586526, 1.2608491437124194, 0.3902467001576774, -2.0649096260514717, -0.49036194849077586, -0.0610117762920781, 0.05452478479851283, 0.37253925094188045, 2.5771316075165713, 2.1409080745444675, 0.862480784326242, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 1.9758885395099315, 2.1409080745444675, 0.20914920379448235, 2.5771316075165713, 1.236304959283367, 1.221524253973729, 2.1409080745444675, 2.5771316075165713, -0.2535979037529452, 0.6903528110169376, 2.1409080745444675, 2.5771316075165713, 1.6410009472602614, 2.1409080745444675, -0.5810557780103073, 2.5771316075165713, 2.004229811649688, 1.3427707685097576, 2.1409080745444675, 2.5771316075165713, 1.6144935628190993, 2.5771316075165713, 1.6016893460902464, 2.1409080745444675, -1.3922536558160117, 0.41222597755567125, 2.5771316075165713, 2.1409080745444675, 1.6105182540451473, 2.181187945679922, 1.4676840794758454, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 2.054109333920704, 1.6874479407126435, 1.9758901270492866, -0.3006900328381996, 1.063412003629913, 1.8608030489806986, 2.1409080745444675, 2.5771316075165713, 3.888759819713556, 2.2291242303142944, 1.718865781192612, 1.9541587858135294, 1.54697625468591, 2.1409080745444675, 0.4514617959866024, 0.8186688685147515, -0.6618015615083195, 1.2116298553420723, 0.6552239131537786, 0.2062348131527558, 0.3369970272502476, 1.3951991130411439, 0.828369267458541, 1.8977569270501977, 1.2370791727304362, 1.1025046118743165, -0.4893129108489931, 0.20645279376728606, 1.3951991130411439, 1.0467058808126875, 0.9659060602346214, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 2.1409080745444675, 1.1165648862729112, -1.345893991280151, 0.8800202340332982, 0.8056945154915596, -0.65085297583911, -0.31513723809576, 2.5771316075165713, 2.172259950587321, 1.066576927476212, 2.1409080745444675, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 1.6509076740287565, 2.5771316075165713, 2.1409080745444675, 0.860103120485817, 1.6906594014341627, 1.105747078655761, 1.8226747104957706, 1.1839017075392546, 2.5771316075165713, 2.1409080745444675, 0.8054562668198939, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 2.1409080745444675, -0.5151684162336481, 0.060560868889933256, 1.1267340375288348, 1.1718930545619486, 1.0203644058821553, 0.8296309997944109, 1.4376453460446765, 2.5771316075165713, 1.475235097004591, 2.067328056244323, -0.4045617451455522, 0.35487135647448903, 0.03181287798425289, 0.8607374260464403, 0.30963743124462784, 1.2755946084836667, 1.2422368541758158, 0.9454730817209114, 0.9512069323103358, 0.9837147551644936, -0.2857733953866959, 2.1409080745444675, 2.5771316075165713, 2.5771316075165713, 2.1409080745444675, 1.021575811167945, 1.5232530770110067, 1.3427707685097576, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 2.1409080745444675, 1.6016893460902464, 0.9534446132122212, 1.5232530770110067, 1.583730774186641, 1.7945512593740431, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 1.7458976103978392, 2.1409080745444675, 2.5771316075165713, 2.5771316075165713, 2.1409080745444675, 1.2411030769346822, -0.391272704958871, 1.6726270329722288, 2.5771316075165713, 2.1409080745444675, -0.8756069982112479, 0.2196023379194101, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 0.7889123176425638, 0.9026901132849322, 1.0624951087511003, 2.5771316075165713, 2.1409080745444675, 0.7297613723607688, -0.43264309808592427, 2.0170107552977674, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 1.6016893460902464, 1.6220872181367183, 1.7417545016752414, 1.3615093013833053, 1.7744611045066339, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 1.3759988949566153, 2.1409080745444675, 2.5771316075165713, 0.7682867102187342, 2.1409080745444675, 2.5771316075165713, 1.6016893460902464, 0.031131723363965275, 0.5251477476767197, -1.0948835847352538, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 2.1409080745444675, 1.3302483817931656, 2.5771316075165713, 2.1409080745444675, 1.261125958303007, 1.8226747104957706, -1.4886726416987421, 1.6105182540451473, 1.1103706546510086, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 0.07930486952856142, 0.5426488676689782, 0.5460926023202395, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 2.1409080745444675, 2.5771316075165713, 0.04211096435339398, 2.1409080745444675, 2.5771316075165713, 2.1409080745444675, 1.3155331159729522, 0.6755644482660174, 1.1110032604734614, 2.1409080745444675, 2.1409080745444675, 0.12099266900952543, 1.1364396529172553, 2.1409080745444675, 0.8246291026630491, 0.8737912294780403, 2.1409080745444675, 2.1409080745444675, 2.1409080745444675, 1.806077354799206, 1.1595850757650321, 5.690357726357057, 0.2045710026105406, 1.2734205014126476, -0.24099196872132464, 1.8079152364344717, 1.3669239226542522, 0.8459263361803891, 1.2755946084836667, 2.184243538570226, 0.2602632572046606, -1.1963171619217678, -0.41801026146731013, 0.547076237678319, 0.027426523347932497, 1.0788479009066694, 0.569134104573232, 0.08882815700437871, -1.3884434588726722, -0.6376306878270861, 1.2867187205163277, 2.1409080745444675, 0.5117992993231685, 1.0962355314854746, -0.15848104361926033, -0.3517310624904122, 2.1409080745444675, 1.4246950060034016, 0.4959377988613425, 1.0353347048449952, 1.0538935624810555, 2.1409080745444675, 0.34649724803090454, 1.8792950594868643, 0.06805084269229514, -0.8736438040087432, -0.8431495367824356, 0.7307369810793395, 0.23597309239067185, 1.5400003854240534, 0.7100352157508328, 2.468636553956845, 1.7663577944222004, 2.3336484609446133, 0.7899618997827669, 0.3508197348976777, 1.662240633162976, 1.4427811443504588, 3.310620792677086, 0.6874006783857747, -2.86863906462726, 0.6807175167538483, 2.3544292747622095, -0.02140449836363459, -0.2698265781209327, -0.5535599131586079, -1.300132061731665, 1.1282420900223276, 1.4496407520509738, 1.116607472544028, 1.2466193879556833, 0.9129955113962037, 1.2479813972819558, 1.721689883816696, 1.8006216270266358, 1.7820328921947393, -0.5834440064903356, -1.0990429720413213, -0.1601082222224194, 1.2030540512663295, -0.08663681066206738, 0.7054678982296876, 0.2541158411770419, 0.40922580500991734, 1.3813249474029958, 0.8692251003171985, 0.9395324238012807, -1.642566608232871, 0.653138684430767, 1.1287152763853878, 2.1409080745444675, 1.5775904625775639, 0.6793979412251173, 1.394206023833639, 1.9557356471747596, 1.8093617632231094, 1.7720119628190047, 2.8343348022279895, 1.247445266624968, 0.8166777274736532, -2.1178801683013333, -0.3285310632585591, 0.059824805934175634, 0.9054452549308267, 0.8766679282270962, 1.6016893460902464, 1.6566810736663429, 0.5340235839339014, 1.6355667371984077, 1.0873732889871808, 1.0698618110062008, 0.5964837083042902, -0.12117256595264705, 1.6629911956933658, 1.6154109657543598, 2.17924242352738, 1.799407544353572, 1.4689434894999687, -0.6530613778315241, 0.5274387461494001, -0.056497549153472576, 0.6004319969296009, 0.6876568291820772, -0.18979790931113108, 0.02259289114369989, -0.9000205644569462, 0.7750853805304944, 1.6016893460902464, 0.657570216766527, 1.0225502725363853, 0.7651933006823416, 1.4279101279952977, 1.5396964511838493, 0.41640222790845094, 1.3798334195243627, 1.0355249561239934, 1.6680451193645653, 1.6016893460902464, 0.3819177566621682, 1.3843939276029935, 1.746996112636467, 1.4450279805931572, 1.6963404994428248, 1.973786449982575, 1.6016893460902464, 1.3843939276029935, 0.7669322599206082, 0.5948202622517699, -2.8889983538040007, 0.8538346371095008, -0.8666041610177143, 0.5128389166766, 0.032852384795119816, 1.5465230774916388, 1.5140166073142265, -0.033369379197756455, 1.063015912018395, 0.891400498523334, 1.035053251339949, 1.145370627615932, 1.1008550343980967, 0.9191748413816192, -1.7058132290725492, 0.8721979802392239, 1.404223822638056, 1.714755648864627, 1.826594364726329, 0.0235427510327276, 1.0867377193957763, 1.2542589018131804, 1.3612314235013099, 0.5059323223976774, 1.4825868664392476, 2.502292257206218, 1.0575484947604268, 2.5568960528323847, 1.1001749280515725, 1.8082328506221579, 1.2238558541485853, -0.2851728893275888, -3.371341492387579, 0.6004684782786713, 0.26706855035842625, 0.6490050888629656, 1.0060799508277218, 1.1879167422331471, 0.8577754758752909, 1.2483408844749364, 0.4919964243293158, 1.440201056401282, 2.530877440018868, 2.899986414430426, 1.2666572369614795, 0.7904820004585023, 1.176071180191594, -0.4305384699311726, -3.66594379650598, 1.7348124593387297, 1.2019524541967697, 1.148344009848001, 0.9105798751371922, 1.0086652723450882, 0.7161745364971419, 1.0363866765904772, 1.2624577196399482, 0.8972890395255734, 0.0639646367353798, 1.4541199721663645, 0.2120571062857217, -0.21305240748055607, 1.5427294792821231, 1.4541199721663645, 1.3851580502811534, 0.04295256165818559, 1.0607405182289462, 1.1413561784919262, 0.49998624792845003, 0.8139519394893668, 0.7608754121433673, 1.9541587858135294, 4.491283633382352, 0.9447193867109897, 1.3965638199627317, 1.1097582741702, 1.5864999133986017, 0.6443591185638703, -0.09105715505289849, 1.0024106484245798, 0.8886314947449301, 1.0024106484245798, -1.0950591694033254, 0.8545604140480593, -2.741696222146002, 1.1197471855069925, 1.3115406544462063, 0.7167778054445956, 0.7112941592927265, 1.977577296387457, -0.06059105247284399, 1.0039050198182728, 0.8431554177552456, 0.28448605125082976, 0.5078221736369274, 0.7651266200125167, -0.0011680571811966967, -0.8913157137985909, 0.9176939257770785, 1.8816066392705963, 0.2443819995451274, -1.7263334526435319, 1.5399464897344728, 1.357468029731445, 0.6263171259943952, 0.8817946394154375, 2.9592403775376264, 0.6796858917417136, 0.7611609979408097, 1.447410854284636, 1.1545038024782681, 1.7425020216924216, 0.6697643222540196, 0.5934520856644249, 1.4496407520509738, 0.6297059580713588, 1.7057215727404877, -0.06195422903018835, -15.245966856742335, 0.37419743836096464, 0.7981892118419583, 1.8246056728131066, -0.30173406295443317, 2.1912208391349317, 0.14619871421549666, 1.6913303962637014, 0.8237927098406769, 0.6379527014216534, -0.08605839346886493, 0.6500958403863109, 0.7606223902821594, 0.7939281364099825, -4.469906455752776, -1.1131916935423882, 0.6707067393305888, 0.5807129514750664, -0.8569863447424458, -1.0091742362911305, 1.3245869921947955, 2.176653418198298, -0.8806762857772167, -0.7674437187490258, 0.7371292085926789, 1.601385323998518, 1.5285929270671585, 1.092972284448301, 1.8095464966480121, 1.9107208542991292, -0.3282218184851131, 1.640964679811152, 1.8227101650296786, 2.8372434575892633, -0.03729440073042237, -0.5790809913490951, 0.20896488893601192, 0.1939654144028182, 0.8018632463034345, 0.08039588436876852, 1.0363951525746926, 0.5044532739230085, 0.7440361103537465, 0.5869890857047164, 1.2556486447253556, -0.2890863776773732, 0.4420757316343118, 0.18687688044274295, 1.440201056401282, 2.1212045351745448, 0.21465078823953862, 1.756326596543715, 1.4134100177676194, 2.230048018062342, 1.6212444063674547, 1.2666572369614795, 1.061307162457765, -0.21407143968827094, 1.632051978981765, -1.7200872415761155, 0.17767163650644854, 0.2570897488066207, 0.7669322599206082, 0.07749455530376653, 0.7669322599206082, 0.4715232106364913, 0.8114479657196172, 0.751019346701088, 0.23597309239067185, 0.36226562214741753, -0.0016291960369145542, 1.7135553773131875, 0.7669927818834399, 1.4744755943450925, 1.880302071234042, 0.9679717015141501, -0.8354407893717367, -0.021493767973412736, 0.030736484673021675, 1.243333201220223, 2.2948015517954046, 1.2264753618088613, 1.3796035486591605, 0.6423179756903321, 0.6071458237894114, -0.7981019867221887, 0.3770870185642987, 0.4398493114130416, 1.393794346690248, 0.7904820004585023, 0.7465423076095282, -0.6007458906398805, 0.06547354364930766, 0.2579176717124453, 0.2600808636455989, -0.46561111423921353, 1.6450575609417084, 1.2527238621016987, 1.6450575609417084, -0.5894200723726251, 0.4802515606549115, 1.474319683509756, 0.1037381553989398, 0.6311532168290374, 1.8263181457590665, 1.4486356540860614, 3.2094720309267473, 1.808645284006448, 3.255544399305438, -0.008867722464607736, 1.044693357000576, -7.068911056207744, 0.30009947972995055, 1.1101594810639726, 1.2399498932632211, -2.167844711772171, 1.6713672844455143, -7.355903097090415, 0.4326675413750123, 1.3099381672630388, 1.8977569270501977, 1.15188968963099, 0.8627583940386896, 1.3123766673821036, 0.8537817437975438, 0.5833860491861054, 0.3575390793285631, -0.14263650840004485, -1.08068743330374, 0.20584300809445974, -7.483836880140203, -2.337081780668119, 1.597767437256235, 1.4966516596079935, 1.7662278137200313, 2.3688405534877535, 0.9867045444492261, 0.7923627913745853, 0.4677704720300614, 0.26994271127561914, 1.2858688780044774, 0.3028278723890809, 0.9119334711167787, 0.17851691166540332, 0.597452573002567, 1.1851113242724913, 0.2022263091814511, -0.5075657444466611, 2.1816643040431587, 0.9157832588065998, 0.5669791829090406, -3.2309738537392194, -0.16826057200110747, -0.04737589053134042, 0.8636495439113756, -0.16944404267985838, -1.941770372595589, -0.26265419338468354, 2.381820172350941, -0.32534249352464906, 1.127772309031955, 0.82236467952352, -0.035523603590389946, -0.7115826267361939, 1.2001606042310446, 1.6184270213311718, 1.042306299973312, 1.229512854560009, -0.19285904747907553, 1.6722909062852098, 0.749796795751036, -0.85306947517085, 1.4621175357713434, 0.835652102157162, 1.034884961457682, 1.6491453211481377, 1.3895030112196194, 1.375264490934813, 2.1320516445775453, 1.1839670637570074, 0.9715979208942259, 0.8851067677186905, 3.127665804877757, 0.32515212689281925, -0.011550246346075238, 1.1395236461813265, 0.5181144667481402, 0.36328186404947066, 1.3504962433462906, -0.07981620839810213, 0.9156943540155835, 0.7327173635328824, 0.4282962048495277, -0.27070810245520927, 0.8482342868363587, 0.6185633424855113, 0.3997119362006044, 0.4259051318268261, 0.48943452919146224, 0.7103122122651163, 0.6718824474410957, -0.726989728286277, 1.195505186880402, 1.4278060589892763, 1.1439832886607533, -5.398460801166593, 3.7080018379514423, 1.0372458925449344, 0.5546630093000592, 1.2278080458144878, 0.9192820638810977, 0.023242131045648373, -0.32979773728976824, 2.183251845564931, -0.3673969798499923, 0.5546630093000592, 0.5546630093000592, -1.0551092657014087, 0.03103967316617981, 0.9411428862628446, 0.7506298933573713, 1.0550719756029494, -0.25899977437853083, 1.4076988997969164, 2.281381559636798, 0.9991021463675516, 1.298864582867333, 0.78164574123773, 0.7951109327461003, 0.8492476485391853, -0.1511941288688, 1.0279824089357583, -0.1146195222549449, -1.983621438074444, 0.5098682980282567, 0.5263407726837764, 0.8878044140183141, 0.884604858305438, -0.07004951573820636, 0.9308290505413049, 0.493822645743097, -1.4134161442725137, 1.1588639241195995, 1.405064226152174, 0.5185794274532136, 0.547076237678319, 0.7030938409685435, 1.0137505319075957, 1.0024106484245798, 1.0024106484245798, 0.782199176152948, 2.369358283210585, 1.7457482819868069, 1.7413499509996226, 1.0199425555268928, 0.5633320267502114, 1.7767637310274964, -0.42277317301055284, -2.838288696566062, 1.2120870444893945, -1.204875182782468, 1.2309710710889614, 1.3951991130411439, 0.460672043196208, 0.0991133246431812, 0.48122679627522624, 4.26726094627922, 1.5731695758747537, 1.4803750661723627, 1.4345450449254413, 1.740932255743826, 2.2258332911243923, 1.5639680718085647, 0.9589824428205904, 1.8961448054770986, 1.5897191991096176, 1.1593148409685325, 1.3475232840185418, 2.181187945679922, 0.595626499014845, 1.3831497895725429, -0.598980156119729, -0.5130653280114886, 1.6148191308583832, 2.2695953055578078, 0.9348395981535443, 1.5282771022472812, 2.076790400037017, 1.6048509951555023, -0.8663067296313521, 0.2702861850372346, 0.8427494511462031, -0.7883337042906612, 0.027376350300527624, 8.9695334272033, 1.811788504229199, 1.4606828904775084, 1.795393271348474, 2.027798302094132, 1.1563966610220657, 0.903548976890714, 1.5865323424929045, 1.6046782921873928, -0.6429106972382983, 1.5032945221933602, 1.5865108035609776, 1.2436032247831612, -0.5759368442341483, 0.18844043390985166, 1.122147961662008, 0.01711279056974943, 1.0174540168721, 1.5056739846204887, 1.1245451287032273, 1.379612649904335, 1.5265121764149032, -1.1374020431256995, 1.5213670663294112, 0.6537838315697102, 0.6231791178442153, 1.153234420621709, 0.027426523347932497, -0.3360828454578625, -0.6981166961503015, 0.4426173627314537, -0.3136188527618463, -9.55894291327697, 0.5546630093000592, 0.3621833100680829, 0.5546630093000592, 1.3023743560628531, 0.41578735315184245, 0.5546630093000592, -0.41457454515877057, 0.5033076614169673, 0.920430384961855, -0.6705972878791898, 0.10024082552204577, 0.6732534235644435, -1.211090449846048, 0.7179340866376042, 0.9581892407696129, 1.1852738032466386, 1.0627903097821214, 1.673191048668923, 1.0384821477866322, 2.3507284997105167, 1.0214265305099035, 0.8997309003856284, 0.07360423090461196, 0.9515145172468519, 2.787852103180528, -1.4309596070741994, -0.24569648536162744, -1.3614718463278852, 0.3868783382142922, 0.680045830532591, 0.8770302717096179, 1.0214669602328894, 1.3657991539908434, 2.145497079696218, 1.289772366348558, 0.6945177387943474, 0.9420093614178757, 0.224929883566971, 0.09967135461227845, 0.7706520417708241, 1.6865156296787316, -0.14617437287539184, 1.5718215666483166, 1.40848055805899, 1.9721838148316528, 2.7069214873423846, 1.249398129436127, 1.9486440307188726, 0.3364716109764888, -0.2613846935948436, 1.26579810890591, 1.531248723605644, -0.4940943716023417, 1.8294346267249855, 2.592419736985314, 1.5596480666517358, -0.2897261465068849, 1.3110589606310419, 1.5371767782724457, 0.7139947988896528, 1.6810873491939855, -0.5529430818555742, 1.7454418339616542, 1.8441646059295853, 1.7877094164475493, 0.8339706302145103, 1.6773781056782602, 1.3118424308779628, 1.5787224265907294, -3.896501815051458, 5.860683792817763, 1.7530390700126588, 0.7508008623095855, 1.1545038024782681, 1.9806762873023414, 1.4527155640041172, 0.5934520856644249, 2.18075036539907, 1.1839670637570074, 1.3895030112196194, -19.43171033172216, -0.8360312163002753, 0.5178563848705646, 0.770758751510208, 1.630084112151979, 1.2725446205626398, 0.9378509020270794, 1.5221685462365584, -1.3852672275570386, 0.33446885722972114, 2.446702411360801, 1.247121738818081, 1.2120963093630726, 1.9022550150869826, 2.162211612332307, 1.3239886428408334, 0.6491769270647729, 0.7980244254514955, -0.0035977238253191712, 0.992611126461732, 0.5948202622517699, 0.310383864966421, 0.36176038246146325, 0.92803855598217, 2.8882151315395466, 1.263271177272318, 1.023399734513939, 0.992611126461732, 0.45099990782452337, 0.96075520401029, 0.7834295150696469, 1.0662189640576194, 0.928400839168046, 0.770758751510208, 1.1432741166869076, 1.0164134092598331, 1.117533627153285, 1.658054271486153, 0.9587592189124166, 1.0793762026689129, 1.0908816477281797, -0.3790147056976685, 0.32844713616755594, 1.440201056401282, 1.2666572369614795, 3.791804081868397, -0.035385175624167736, 0.9901289189434542, 1.0024106484245798, 1.003900543886815, 1.3254906354931608, 1.4580835265497782, 1.9466946718100544, 0.8575115941521162, 0.09030040294281372, 1.071349357582088, 1.7561688416504204, 0.7458717647665725, 1.2656896113640805, 0.8028445499544457, 1.8424378945538185, 2.609334195543554, 1.6013773805263287, 1.6391259626974386, 1.1545038024782681, 0.5966687679349251, 0.8417786810803572, 0.7437889380559172, 0.8579866161203838, 0.2832363382076929, 0.6866019575635715, 0.26706855035842625, 1.0024106484245798, 0.42550015635190563, 1.9555906067767395, 1.0451838828802817, 1.6655930485113528, 0.8730465800313031, 1.8006216270266358, 1.0295302216882618, 1.3285995274304996, 0.7462900062043808, 1.1639727680397622, 1.3206120870503204, 0.2333622483548818, 0.5976392563635606, 1.0509409746406928, 0.5181144667481402, 0.20696647720574832, -0.9120427970172754, -0.016008365398709673, 0.016917341215200804, -0.264448122684935, -2.5061671451540652, 0.9386096558347454, 0.8091130996068324, 1.4679605450443816, 0.9106092903900619, 2.230048018062342, 1.5446538067747213, 1.6779949336796516, 2.146321903597431, 1.8534689712067784, 1.672363808253149, 0.3747669264784066, 1.4557661513629254, -0.2507086847418548, 1.4217949121901121, 0.07749455530376653, 1.0961111734451094, 0.4230680091764315, 0.8936583612520513, 1.3001277792906027, 0.12555721680412263, 0.5059329568101895, 1.0024106484245798, 1.4671073742363123, 2.592419736985314, 0.08502200794026599, 2.3376859987747776, 0.42550015635190563, 0.20854041531579653, 1.000836120999969, 1.07838413461373, 0.7194054460291189, 1.54697625468591, 0.8796007299174804, 1.2723706995090576, 3.3678146788507854, 1.2216636772157397, 1.1722675385504588, -1.9533166328545517, -10.717205820728964, 1.065460189877128, 1.502193127460666, 0.27837588556179116, 0.8668216740974669, 1.0535656947466574, 1.4281036366237092, 2.9726004101271286, 0.14210952239293614, 0.9266514627553889, 1.6322998603758396, 0.9019239494075438, 0.9089466340137741, 1.0578016783000475, 2.8108366069826616, 2.694165183137947, -0.831057694834831, 1.2223324937229174, 1.4692253820074541, 1.263667394545511, 2.2667357959159338, 0.14851499291905365, 2.9190954853635525, 0.843521280876357, -6.726252473780642, -2.564128538564286, 0.5736285535201765, -4.479543298307246, -0.27375253565706803, 4.955361348912369, 1.0875757818542877, 1.6929430502559886, 0.6004684782786713, 0.19501839552294772, -0.7216480860627476, 0.6564548099932276, 0.8572978501938543, 0.5546630093000592, -1.7166466617892653, 0.015976550851866195, 1.797861831848256, 0.8943719132414151, 1.3269818264499025, 1.4610753173528879, 0.4192075642880183, 1.6439878563594825, 0.11470115960240282, -1.897825626504286, 0.6340479237688935, 0.024065851569339797, 1.6016893460902464, 1.3858815176858876, 1.7458976103978392, 1.899993527503598, 1.1783306025138653, -0.008635693702680053, 1.9793687654796837, 2.3106115192215055, -1.3372790348711143, 1.394641139551192, 1.1015834247244625, -3.1003316289521545, 1.0809526289686064, -0.6147233635041559, 0.7732724769244246, -0.14303447988756549, 1.177563331771757, -3.8201692312972364, 0.28470001781924353, 0.5982075491050418, -0.2874842704456416, 0.7834295150696469, 1.6016893460902464, 0.650436775110611, -0.4998522104909021, 1.3385575669398337, 0.430640500469592, 1.398727066531771, 0.4939989253612064, -0.3104074690967722, 0.6030484758380222, 1.6016893460902464, 0.5260816640262648, 1.3844939325504668, -2.8598220941022383, 1.2351291854862043, 2.2208821906499865, -0.34925793430806273, -1.9136131789947584, 1.6463431454176136, 0.7864163000320424, 2.127903952178394, 2.009279484593676, 1.229386889207274, 1.1698095449667112, 1.660192907313073, 1.8643366607064793, 2.182996486456691, 1.3021437651335945, 1.3615905243353341, 0.6717626058040743, 0.4770967131771928, 2.3298916323438, 1.5980940801604617, 1.8910586544388375, 1.823797421380788, 1.4136123550480675, 0.04299938581569099, 1.7920018156844943, 1.694078536073811, 1.89977468459775, 2.25761164957635, 1.778663462731866, 1.1094059866732393, -1.873572745492835, 1.1808809668983442, 1.6802300775417003, 1.421019303771786, 1.5563754634283995, -0.9771288015671665, 0.9679079170761004, 0.8271978100011933, 1.8253868789977343, -0.04113433987875595, 0.7296394766743234, -5.931049900811777, -0.3703420134655586, 0.7667312303923809, 1.2485912961647847, 1.448230310927882, 1.6016893460902464, 1.199397887366839, 1.1613655836378014, 1.5993136322366515, 0.9307779811356074, 0.467551526043968, 1.4424029093503943, 1.6561947472943206, 1.1825061850229852, 1.4483414804309127, 1.9345059711647188, 3.104845204253331, 1.4483414804309127, 1.0605883145282686, 0.7307157496365235, 0.7814757031163982, 2.0880106910963736, 1.1324650889719667, 0.5250896400510174, 0.6858859651732717, 1.0375628942242503, 0.8864024450663123, 1.4072009663797431, 0.4659076675456122, 1.335938119645981, -0.13654862309304663, 1.1590137019453943, -7.0414946682386965, 0.8919706387780162, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 0.8919706387780162, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 0.8919706387780162, 1.4832838572401548, 0.8199266065022739, -0.03300330462486703, 0.8749961319115276, -0.8490311154717071, 0.5368459007622202, 2.0230105225996424, 0.38565392376232105, 0.6246773241076878, 0.5045627936866749, 1.6527774250332112, 2.014971292342905, 1.9443592329672923, 2.357238393884624, -0.8840554052017077, 0.6409699846506748, 1.0029355387015284, 0.7774695230785466, -0.017879832151108416, 0.5572408792526004, 0.6582249374071268, 2.3866304374976077, 2.0230140920088537, -0.6101786748333881, -3.0471412225625443, 1.3825910903970162, 0.9832635683017039, 0.42045229925630867, 0.6537929514249402, 1.2384129414993172, -1.8619659593482638, 0.5054919513232249, 0.7173027494859594, 0.4645434609283058, -1.423704102514462, 1.5945814856652762, -1.2210081407218178, 1.4487638773145335, 0.48292103901383154, 1.1848131290086819, 0.41197712093177186, -3.9460634279040923, -2.0272078518035377, -0.6931443707188096, 1.7660093760261169, 2.239667625703868, 1.6798141956665769, 1.9348182406349852, 2.6757747367780977, 1.5920913698925074, 0.23597309239067185, 0.7049350703941581, 0.5804447458926473, -0.7188395631509134, -0.9455738295683126, 1.1866680611184253, 1.3238732470051702, 2.4207368530509514, 1.2392741146258852, 0.9651237991706526, 1.0326893765358567, 1.5456331484475634, 1.4996775219337213, 1.1345064950470476, 2.0234895801989814, -0.12712034748722403, -2.1165205688159094, 1.261329312683894, -0.2587649197240598, 1.8626133382172816, 0.19245662758536305, 2.6248204773914634, -0.43139467353463323, -0.13769606625856615, 0.5546630093000592, 0.5546630093000592, 0.5546630093000592, 0.5546630093000592, 0.5546630093000592, 0.5546630093000592, 0.8388437437785294, 1.4368276268057107, 1.8248403136122227, 0.6656238263231856, -2.3035488586596617, 0.5546630093000592, 0.5546630093000592, 0.1560082913416307, 0.5546630093000592, 0.2689675436336389, -1.6550525248934889, 0.7016759472172919, 1.187225831114718, 1.7312812058741143, -0.009877121519436122, -0.6113848137159116, 0.03151944431498868, 1.243904747015427, 1.6719071570003299, -1.2997273606684827, 2.4629607624048107, 1.3679523482089846, 0.9314368569577502, 0.0576448111201808, 1.336310717275091, 1.2580406862299642, 1.2205967853236375, -1.023892520929296, 0.13113052312616275, 1.0401479328436685, 2.764670225168777, 1.042505875407836, 1.0930447033731754, 0.16433879854111325, 0.22277970960541055, 0.6946486706980003, 0.7988646470470717, -0.16793643323827148, 0.4107258700615124, 0.1524465401325433, 0.49761100985752105, 0.49875307107097033, 0.25351127602572354, 1.3134520761902018, 0.5260246296389756, 0.47111311357773344, 0.977115793797005, 1.196173986797772, 0.5685783486655279, 1.6591102631127854, 0.5666626218554519, 0.7520418821845892, 1.3039070221136944, 0.9415701156708074, 0.8063263718630013, 0.9879640947907481, 0.5229141151461323, 0.4914479908702962, -0.24640500917349503, 0.9785937922837746, -0.027747507516249936, -0.3830418642510015, 1.1179947025778123, 0.760938878722366, 0.9425261363543592, -0.8359216551201433, 1.3474696372908432, 1.2576808833473525, -0.3873551132530778, -2.482517280113728, 0.5184616348400514, -3.5297998817935827, 1.8096594762760885, 2.274625971318841, 1.3112539977891198, 1.9709794956537057, 1.2396601248881864, 0.182686697170906, 0.6064181497004442, 0.23597309239067185, 0.3524178940862146, -0.029179275545060696, 0.47439194626248204, 0.0492635515338333, 0.7011532550921636, 0.10084119117932003, 0.5546630093000592, 0.5546630093000592, -0.81416224906123, -2.6562263024861696, 1.6436567501307573, -11.790533053688863, 1.4689516451064748, 0.5873626242459166, 1.0108212419657823, 1.2858617865559454, 0.5507723053299426, 0.3009758010407444, -0.8749577112193124, 0.872677589429581, -4.76402681216739, 0.9940837656604965, 1.0410455187148846, 1.8662282037477318, 1.9541587858135294, 1.442207150239991, 1.4240857762951746, -1.3663279548593845, -0.8477313757038699, 1.2539337142851332, 0.9229047298813632, -0.7818344811012922, 0.8536376832364462, -2.2421277916501436, 1.0454231404273762, 1.4918954702752383, 1.4683058432780753, 0.9086723912242124, 0.49633250900032344, 0.12575615808206148, 1.5459019519342079, 0.924608492587518, 0.970909111213779, 1.6016893460902464, -0.9824115445825631, 1.116607472544028, 1.1492053530373532, 1.1643086990884017, 1.116607472544028, 0.6018819934556305, 1.3145468767883597, -0.41801026146731013, 0.4796418373339779, 0.30287523126710925, 1.9550875638559448, -0.27620500505797974, -0.018006467119532438, 0.8560499051666016, 1.8884228809228105, 2.030030157269158, 1.6249915034548283, 1.7960552071682252, 1.0529370915740834, 1.6379242822357538, 1.1297145395536266, 1.645270955899604, 1.1184218380200739, 0.36303517669111385, 1.70071438384149, 1.727401464140664, 1.847863579131644, 2.031682964867173, 1.6997391564779372, 2.3555987359799344, 1.2312982456107373, 0.8547023320690429, 1.9554595951509937, 1.2918414531622209, 1.6710591257593121, 1.541725089705426, 1.837507338235618, -6.5311839135648615, -0.9469742951279442, 0.9319544336002349, 2.6971169521029, 2.2120741935567487, -0.40364075963388524, 1.5198327084565442, 1.6856202237429336, -0.3472725354148798, -0.1856981776851356, -3.9796937749018375, 1.471205719776848, 1.955423241176609, 1.1112385226413688, 1.8015303644936917, 1.7195545618560306, 1.010765327929042, 1.5754537554569494, 4.170127643176943, -0.2755523724227872, -0.1403598844543084, 0.4842231276643414, -2.9831015277222828, -5.585692328519823, 1.402723277354274, 1.703568287363808, 1.598171758583052, -0.9521686939601938, 0.013775165633712838, 0.7567542328888398, 0.7881390154693648, -0.06377224629206327, 1.2860073217283499, 1.2718937989501293, 1.5158037499300403, -0.043338199703747776, 0.6004067882528281, 0.9569118258173029, 1.4816904294489088, 1.2267182094755258, 1.7221623054834965, 1.6444160001117418, 0.23597309239067185, 0.672273305437156, -2.049407777015004, 1.7084262824393879, 1.0085524009021711, -3.5831513954311607, 1.6879817468492209, 0.23826851669996096, 1.4018570704187814, -0.41411202119210244, 1.0527057195024612, 1.6450575609417084, 1.326677306657812, 2.3420420387763676, 2.129602322512003, 1.6450575609417084, 1.0047136819331808, 1.7091178498802864, 0.7942149342892014, 1.6450575609417084, 1.4199436237296863, 1.6450575609417084, 1.6528749887619973, 1.9519886796028572, 0.5026951189185066, 1.6046026871020724, -0.5410801268473138, 0.68921157172862, 0.9837427387456811, -0.5805143779523618, 1.3151077116798033, 1.2786674825156108, 0.04131989317923404, -0.7325233565298713, 0.4500136249434755, 0.2143556363563404, 0.6422890222485999, -0.4989138562727867, -0.1981536669458334, -1.252767203340791, -0.5902963122790702, -0.7473357997514001, -1.333198661479292, 0.3930126121220504, -0.08445857196401677, 1.1414424541292858, 0.7330983154165179, -0.25973186397672143, -0.8917435679057354, 1.597017743645137, 0.29883392662811087, 0.3992934416175524, 1.3184115072997764, 0.6219808756184637, 1.597017743645137, 1.3329257852103653, 1.4822380830845676, 1.6557117567390998, 0.5493378024776595, 1.8017408404023727, 2.1327245426898527, 0.19871424258948175, 1.7670548467289906, 2.0115702319075073, -4.24820405542355, 1.1472599474996257, 0.556373197163949, 1.969709809753148, 1.4097286064869792, 1.2610089636184514, 0.5030451965587404, 1.3036060584237747, 1.8539303086380323, 1.2447426029448272, -0.11265709455407971, -1.0985740448079908, -0.23269771787501128, -2.39344439080409, 0.392337163135579, -0.25272867516323105, 0.775247998983655, -0.18113071171600106, -0.2969386338508738, 1.900001088227931, 1.605331576241366, 1.7055372859834388, 1.631006635126536, 2.068323207720261, 1.817395965562862, 1.0382459158801935, 0.8761678223912848, 1.2331687705602823, -0.3775388105142655, 1.2331342888896226, 3.6497057280897685, 2.851832702745311, 0.4359268362688431, 0.24165954133086373, 1.0957205947719908, -1.0008070664251925, 1.1379786784078967, -0.7774829481440239, -0.15852815458024175, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 1.4832838572401548, 1.4832838572401548, -8.032985997043664, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 22.925575210841075, -2.068958373879884, 0.5400908190860035, -0.11758627950422287, -2.764843940347336, 1.3265261331461664, -4.602517726594472, 0.9455066533039644, 0.6508379536449495, -3.7557469070892933, 1.3531971883133804, 1.2238530372362209, -0.10688644415033496, 1.3752575231039976, 2.589716489962258, -3.3542712220444253, 2.038888408135057, 1.1619247219910425, 1.5351039407423548, 5.241719346633542, 0.9668517876722367, 0.4030398106069922, 1.7325082046496896, -0.8260007214334527, 1.558313123830409, 1.2180541604002673, -2.081262116234143, 1.3700873252941448, 0.9957539847756399, 0.6245122751443699, 0.2876272797259328, 1.6553970021437086, 2.0365809440908826, 0.8210041280435052, -0.1747414847938273, 1.6450575609417084, 1.9999056743189192, 1.667568091857955, 1.2966862808334754, 1.0589770689966254, 1.6450575609417084, 0.22836425224144832, 1.6149681344708753, 0.39642298461152314, 2.3205863967709863, 1.6487462599758969, 1.4370890764726012, 3.0684310837617432, 1.6450575609417084, 2.320231299331322, 0.9959649263735942, 2.5396084150853695, 1.6450575609417084, 2.2337045093099586, 1.9084402103231701, 2.12716069492192, 1.9632062852691856, 1.6450575609417084, -0.17285338593508212, -1.4513268234238954, 1.4483414804309127, 1.0894018360310669, 1.4483414804309127, 1.828137563184473, 1.4483414804309127, 1.6192747487572972, 1.9341899232564832, 1.5325470302962476, 1.4483414804309127, 1.2829806631184788, 0.6226932035754066, 1.1694935062799376, 0.7284994043968638, 0.6350113039522969, 1.4516575519834334, 1.1991896977322964, 1.695690285082407, -1.7250932886621997, 1.5743131653460491, 1.4028128964301088, 1.4992143244121143, 1.8562561947039367, 0.46547140802523235, 1.0813458312443354, -1.2123173505814582, 1.6450575609417084, 1.9503746847747976, 1.818568922239871, 1.863290766982397, 2.46065891508907, 1.7746567678750182, 1.7791569016626856, 0.2212865245299948, 7.663707199665017, 1.1079625979843486, 0.49292883591149395, -9.546037606490781, -7.523541221830094, -7.488418828354674, -0.27746022634255796, 0.005841089469522748, 1.740436492900937, 0.444564120043514, 0.8732701275920601, 1.299807039072673, 0.4521714123974707, 0.9421537454241787, -0.7350841852307097, 0.8341580475627939, 0.645071613595637, 0.8919706387780162, -25.701987929622703, 1.9271383210425999, 1.7588452763871394, 1.0778126797808902, 2.6536546164160697, 2.021392474178244, 0.3461802440140952, 1.592825955395132, 1.2852330645909913, 1.7852304039751712, 1.105595671045962, 1.4436757001693836, 1.5027874479275398, 0.7843883535203987, 1.7517750146531297, -2.67463093931152, 2.0143889716691814, 2.3472400587823694, 1.4749573920587944, -0.8165581547653263, 1.1322721830136346, 0.9301331088559726, 1.4483414804309127, 1.0770000562282054, 1.1825061850229852, 1.4483414804309127, 1.5803927649786431, 1.1004151968661489, 0.10893120014415886, -0.3554208382280989, 1.5877860665046732, 1.290849121703342, 2.0964468389786735, 1.0576691904665907, 1.6001446025214479, 1.2663779360789527, 2.1701482963617376, 1.863610115077241, 2.343326315082148, 1.8215543576824158, 2.4970329885289235, 2.427809155131393, 1.4142531516436199, 2.310667946296234, 1.453625438332704, 0.43057504016677584, 1.0203954382982117, 1.7044052830952672, 1.2405439319715104, 1.0171195273283307, 1.9798559465250012, 1.7579891000803736, 2.2838436671999247, 2.048479148988388, 2.0372203197138115, 1.7820594266068484, 0.9531288545281564, 1.685230627139479, 0.6618029581561439, 2.2599184849093232, 1.2525605448392478, 1.8493526259815345, 1.8738104112494087, 1.5030605545926605, 1.4167247608788613, 0.857481678350734, 1.4852360862591787, 0.29753718943287966, 0.7052733346454517, 2.54804783515491, 3.7571082978032426, 1.9401704933269583, 1.5811911079055025, 1.989857827949144, 0.9692910687603306, 0.9961321235291803, 1.4026495373062249, -9.129409919968849, 1.8904682028190942, 1.4840348972489996, -0.36961964009174597, 1.6384286170680658, 1.5012673926737796, 1.6056689826211157, 0.5238109607185498, -2.652423321155612, 1.6450575609417084, 0.8032656050872917, 1.6450575609417084, 1.4178625729092236, 2.5448660120173425, 0.276433890735978, 0.7168919575387077, 0.9657201466655071, 1.0878094202813338, 1.3019382727237387, 1.8738104112494087, 0.8620331095348893, 0.10327436498155071, 0.903053848188235, -0.08661914943214713, 1.1523667182171005, 1.333620823251918, 2.0329467878264342, 1.7956674524149674, 1.7306855353147543, 1.300788142533094, 0.31155053230643803, 0.9127466305723416, 0.42240726537782747, 0.19018398924813482, 0.7779790424143275, 0.9329484133413685, 2.2492153895640814, 1.6450575609417084, -0.14484637387501356, 1.735711465090068, -1.6194943922309533, 1.6970877447810129, 0.8079701653883574, 0.7616625221709584, 1.6450575609417084, 1.6450575609417084, -0.11288544304001788, 1.590680361712782, 0.4739128634117522, 1.8723723286133673, 1.4483414804309127, 1.0649390944685626, 1.4483414804309127, 1.6056796596631724, 1.0834468300326647, 2.177369652097237, 1.439084412016256, 1.6221908356787569, 0.6148531205004112, 1.845787455542807, 1.919067205007239, 2.110445163086786, 0.21815237777387456, 1.2008503673331183, 1.5115585870421633, 1.7119288100219208, 1.7789029559812997, 1.9279159463431867, 4.476636029228178, -1.4192795310095914, -0.6214004635401683, 2.0538429455794764, 1.0998387810830392, 1.1383786979174573, 1.769406997500476, 2.6219178301585977, 1.0283474226622182, 2.146057755840769, 1.563067472047909, 0.3228059282754656, 1.5922097593792934, 0.9546816789137277, 1.7616330678338963, 1.3692982191334697, 1.2985890732026277, 1.6859125364193226, 1.6619825406405029, 1.2026079967075463, 0.5715144297181879, 1.6622532439186761, 1.6450575609417084, 1.596875760604279, 1.6450575609417084, 0.7303710166212094, 1.5058285472780124, 1.7357692442002215, 1.8496513962269274, 1.266354079904842, 1.5985617408277653, 1.8661842967044646, 1.6450575609417084, 0.9270125129716498, 1.13573955214933, 2.0780144303956867, -1.6326761381621504, 1.4144903277676961, 1.7316612793966224, 1.3061726411983785, 1.737555811583639, 0.7989887186596363, 1.739116503404321, 1.8796184032231273, 1.9582842667679812, 1.0964159847070267, 2.4274160410912713, 0.4643635551206017, 0.018328473554598934, 2.3706468574415362, 0.7664842746706382, 1.1988493397709463, 0.955341911958506, 0.00491514115376121, -0.9581214013051454, 0.4110437732545187, -3.0366052036668143, 1.7674886927099203, 1.1509822235496612, 1.6597613239744176, 1.656429728635327, 0.05112044493157339, 1.6332051894497284, 1.8030515577913893, 1.6158763840800185, 0.677221436998343, 0.8888900182332932, -0.46274753143462555, -2.630013195967103, 1.6450575609417084, 2.785918106299133, 1.801537137807673, 1.613113306676227, 1.6903977740137324, 0.8000277981593975, 1.7986604669294275, 1.6450575609417084, 1.9155350160737754, -4.604900014288362, 1.7397860438233872, 2.015069346869579, 1.3963582640290138, 2.0794675497140918, 2.142003599580714, -0.44025255869764035, 1.828247832371407, 1.7816046997463595, 1.7540047099903047, -7.5337934090073215, 0.1375402417501824, 0.8656718521580926, 1.857550066726713, 0.9816836476800528, 1.4483414804309127, 0.176950732021975, 0.21987630693224877, 0.08434334851397715, 1.6450575609417084, -2.3105208041034553, 1.6898313932728954, -5.748670818900123, 2.5239168038423356, -2.6211475737089236, -0.06867258996748024, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 0.8919706387780162, -1.5422813536261006, -33.685229627268775, 1.9478902495164399, 0.707797381816985, 1.7034141468672928, 1.9478902495164399, 1.6393825742951147, 1.7034141468672928, 0.2676218925746915, -0.2742165331756423, 0.00174022610440408, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.7904459889236815, 1.6476841571545717, 0.22988336927827363, 1.660702394735903, -3.458446580140627, 1.1116925093328647, 0.8394746792824844, 1.5630966767967884, 1.2429923543207262, 0.1877891761769738, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, -3.893201871853421, 2.2208821906499865, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 0.9428316673603538, 1.130332286195984, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, -19.335368801688425, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 2.2208821906499865, 1.9478902495164399, 2.2208821906499865, 2.2208821906499865, 1.7034141468672928, 2.2208821906499865, 2.2208821906499865, 1.9478902495164399, 2.2208821906499865, 1.7034141468672928, -0.7638824346921861, 2.2208821906499865, 2.2208821906499865, 1.9478902495164399, -4.669991384592111, -3.8747523930089414, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 0.8919706387780162, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 0.8919706387780162, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 0.8919706387780162, 1.4832838572401548, 1.4832838572401548, 1.1040621030716513, 0.8919706387780162, 0.8199266065022739, 0.48158089008973437, 0.8919706387780162, 0.8919706387780162, 1.4832838572401548, 0.8919706387780162, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 1.4832838572401548, 0.8919706387780162, 1.4832838572401548, 1.0641164942734291, -0.8377104706280415, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 0.17246047859676708, -2.4911360219857714, 0.8919706387780162, -26.06690230825213, -7.888557321420688, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.9478902495164399, -2.5714138329178855, -1.9165779584748404, -7.262375872422224, 1.4832838572401548, 1.4832838572401548, 0.9554442535574039, 0.560908383913958, 1.4139564693431632, -29.558841215686, 0.8959938989426485, 1.7034141468672928, 1.9478902495164399, 2.127903952178394, -10.100117766307205, 0.13838577575373026, 0.47069192054374487, 0.8919706387780162, 2.5835409036317625, 0.5807919654936091, 1.4152900206398815, 1.2479335638404938, -4.2418749794647015, 0.7338459038884311, -0.2517155871406883, 1.6450575609417084, 1.6450575609417084, 1.6450575609417084, 2.234640271073596, 1.972499206158037, -0.07743751384381672, 2.1510827974493503, 1.2577128379162696, 1.0419017485926958, 1.685731542135588, -0.753260594172395, 1.5022284846992575, 1.4044311223105894, 1.216741833617956, 1.2755114186632668, 1.7842745978878651, 1.7757126034591022, 1.6866335728795636, 1.080779547387475, 1.8258882571307966, 0.04785232236197201, 1.5561041311456347, 1.525516669168525, 0.7789347174399641, 1.3568446378088155, 1.3154552991893655, 1.7353411233032252, 2.381309378816203, 0.6264360470814205, -0.11432287942776062, 1.6592762353261878, 1.4353631874710406, 1.6382586168061053, -3.555532630248865, 1.52560644764697, 1.0072109267750573, 0.9944974810260393, 0.9885843133663597, -0.6732747841354505, 0.2398468526641763, 1.7334457066683904, 1.9252550007481959, 1.646313738364966, 1.0579852432806025, 1.6450575609417084, 0.7218478507186888, 2.415262350780759, 1.0926252994339611, 2.0480721479088455, 1.41353430984329, 1.6450575609417084, 1.6450575609417084, -0.1792517158658394, 0.6260496995690074, 1.7944931796579935, 1.6985647938982509, 1.5863338488287342, 1.3419173035952763, 1.6450575609417084, 1.6252950043738579, 1.6441798168908808, -0.0577898310430289, 1.8071232542135818, -0.016919652757509146, 1.5075177438139773, 0.47624333202251223, 1.6450575609417084, 1.6450575609417084, -0.05437991299819425, 1.4289964663579198, 1.3256858546078245, 2.832134487974841, 2.226878201521533, -6.585752018624245, 5.419404204386505, 1.9253234320219976, 1.2391310287563717, 0.5563456146876486, 1.4689143600176289, 0.4033177480578348, -4.392671776993276, -0.8019979897605545, 1.3875321719781426, 1.9970318204554345, 1.6443375233768966, 1.9592327392089823, 1.844843844416604, 0.7866039552977391, 1.2905156090870422, 1.5301658107359917, -0.016177581228698917, -2.9491540102209926, 1.6450575609417084, 2.024807643473388, 0.9853670033685683, 1.6450575609417084, 1.6450575609417084, 0.28275881058624486, 1.6450575609417084, 1.6871346389367228, 1.5746384180416102, -0.29135525935698514, 1.423251654394836, 0.3210605232981807, 1.4081384678874298, 2.5111926424781643, 2.3839924438686166, 1.502841236698649, 1.1691669840764791, 0.8943590711645356, 1.0280588995476394, 0.918625209485804, -0.2659032717508921, 1.7891747537708491, 1.5299889079684401, 1.3891983101578573, -0.1639129306340845, 1.5928062348625907, 1.8734424486140249, 1.9995395339024487, -0.49657526771335336, 1.6450575609417084, 1.8264169121618241, 0.4369632370832309, 1.0486147441587326, 1.0669224257207843, 1.6820577339264127, 1.0820133301811468, 1.6059338641431238, 1.6281970521322544, 1.6450575609417084, 1.7700944446612674, 1.8331791245046383, 1.7345639080340518, 0.6560184515157523, 0.6055126705817299, 1.6450575609417084, -0.23469614254647322, -0.0671497146756306, 0.03820235042482131, -1.2708920855083745, -1.2752651401533384, -0.4123948866340844, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 0.5166103005419976, 1.846375220364581, 0.5607611683229742, -0.3347516875028964, 1.7034141468672928, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 1.9478902495164399, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 1.7034141468672928, 2.2208821906499865, 2.2208821906499865, 1.9478902495164399, 2.2208821906499865, 2.2208821906499865, 1.9478902495164399, 2.2208821906499865, 2.2208821906499865, 1.7034141468672928, 2.2208821906499865, 2.2208821906499865, 1.9478902495164399, 2.2208821906499865, 2.2208821906499865, 1.7034141468672928, 2.2208821906499865, 1.9478902495164399, 2.2208821906499865, -0.05610946389883428, 0.14224414791294324, -2.053970439965017, 0.7934078843330218, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.1616138155758367, 1.0001434583269475, 1.9582981257316392, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 0.9219125334796858, 1.4799807941322216, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 1.9478902495164399, 2.2208821906499865, -4.023106686179713, -3.985295137617634, 1.7034141468672928, 1.9478902495164399, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.9478902495164399, 1.7034141468672928, 1.7034141468672928, 1.7034141468672928, 1.4799807941322216, 0.9942955299937667, 1.7034141468672928, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 1.7034141468672928, 2.2208821906499865, 0.5149801227151335, 1.9582981257316392, 1.6393825742951147, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, -0.19933143020965052, 1.7034141468672928, 1.7034141468672928, 1.7034141468672928, 1.7034141468672928, 0.8431041162708791, 0.9219125334796858, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 1.3264843446555512, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 2.2208821906499865, 1.7034141468672928, 1.0293531474856463, 2.09282062281143, 1.6590023265452305, 1.6872757410626318, 0.5339729209743114, -3.7301382986756235, 1.6590023265452305, -0.6632764619986296, -0.9060760682079855, 0.14006767962359504, 1.2737836314166788, -4.498538717289632, 1.2810306756834993, 2.391562351204694, 0.5550186100727503, 0.8921750837543333, 0.11129680520486313, -0.8514717272815463, 1.7414280222527003, 0.26163648230470277, 0.42422813345560584, 0.38279827537792543, -0.8065758336964719, -2.9444553781156912, 1.0351352687589135, 0.7278729281758608, 0.38933102997164737, -0.05366862061446259, 0.05333024287342167, 1.7122521393161754, 2.1005192067476783, -0.008520369262133592, 1.6199946435359267, 0.07957560399412253, -0.0901549172323727, 0.7370570690110515, 1.1688523579627945, 0.2374273431278187, 0.5633629270590599, 1.5086261143732098, 1.658088092022328, 1.405827493821015, 1.45845516554643, 0.28447491925256163, 0.9836337695233891, 0.46020269598001107, 1.0099184295460315, 1.7191044487597456, 1.597017743645137, 0.8356267865055059, 0.919644764227447, 1.5510315532670464, -0.18441471930052736, 1.3195538080298321, 0.30004544665213984, -0.9849344566876602, 1.7648006091248332, 1.436937718342343, 1.03544714821681, 1.7648006091248332, 1.436937718342343, 1.7535028617429576, 0.93684177135637, 1.7867049293543595, 1.1818285930992076, 1.7043771026583798, 1.8710342072385076, 1.8832368845252268, 1.1761406563219117, -0.8291127647278965, -0.209928450983958, 1.7648006091248332, 1.597017743645137, 1.436937718342343, 1.597017743645137, 2.0194768912319274, 1.740863983930548, 1.6102338621894592, -1.591767029662961, 1.4461042953632808, 1.359703909329484, 1.3282803699379335, 1.4463633029948406, 1.2405133433882412, -0.8604996935489005, 0.00872891593284117, -0.7511602023433807, 0.1098466564054226, 1.7648006091248332, -1.2361670511333434, 0.9191063242764377, 0.9750594763081984, 4.6475167238425925, 1.4363176390399055, 1.4855261688894716, 1.7341450765547426, 1.6398077956743455, 3.0768579450046327, 0.9006304334487364, -0.06472156104292359, 0.8065019398531749, 1.0222559595226608, 0.22072960868118177, 1.7668664882745768, 0.8099350174085527, 1.436937718342343, 1.7648006091248332, 1.436937718342343, 1.5745425129746575, 1.7648006091248332, 1.3325422021016222, 1.352229228794558, 0.6193346266160601, 0.9183785878764579, 0.9403766745993818, 1.436937718342343, 1.7648006091248332, 1.5282457618106549, 1.6934094667788384, 1.5059306263843724, 0.7633594507098967, 1.0627105459459476, 1.4684463487589057, -0.39336653916119924, 0.6594744843748016, 1.7019917384275554, 1.436937718342343, 1.7648006091248332, 1.7648006091248332, 1.436937718342343, 2.1793784912676353, 2.03580091686478, 0.593930819425231, -3.282406291238874, 0.7511831745711539, 1.326534125352303, 0.8525967505681507, 1.5578984497943928, 0.983634925544315, 1.63440240105647, 1.2939036323953694, -6.75799181036307, 0.5697287804657262, 0.3290041878812092, 1.7648006091248332, 2.0206256609081295, 1.9929128471037763, -3.635306957177578, 2.06186816012059, 0.43743972772991013, 0.042382903285125145, -4.251354524778189, 1.0874086040874609, 1.5400506166486314, 1.1616138155758367, -0.6411034709108888, 1.2311113710826478, 0.6815640424722157, -0.15088915739611378, 1.4768783889005246, 1.96778138368807, 0.938948284591708, 1.110836976455737, 1.96778138368807, 1.192267269994357, 0.9445171465687056, 1.2187333805500384, 0.05677547945394065, 2.0003422480363953, 1.7906227318313772, 0.6492309612402636, 1.5527030965018216, -15.80363335320018, 2.548519250294246, 2.4956466770177945, 1.7292262302203876, 2.2537651796351357, 2.059843553800024, 2.246584002401516, 2.2844162357619875, 3.242198562560646, 2.108332764125251, 1.8196234136304976, 1.3099320547589892, 0.28238755911446456, 0.9000143717291567, 1.3792505674156892, 0.4051749504209874, 1.4845056800300138, 1.1685294929883223, -0.3556829171764655, 1.8884602740160736, 2.146043003705446, 1.773574835172481, 1.7620781560483152, 1.7820594266068484, 1.8089706887136887, 1.6227537185493257, 1.9401801380386554, 1.5172950372127345, 1.4942762869187216, 0.7731409464054867, 1.1087631922004728, 1.6000297085155204, 1.836146355315879, 2.011836700729283, 1.6089596255282428, 2.1115091680440496, 1.485600606903986, 0.9727569178979713, 0.9136163047072378, 1.7960552071682252, 0.6140794512972064, 1.7960552071682252, 1.0381247452910414, 1.8369916008299698, 0.9035034243651042, 0.28909908499724535, 0.8298026373429769, 1.2145538714668445, 1.5367336211700775, 0.22407140918379984, 0.6505173004799603, 0.5462855372129318, 0.7524992794818582, 0.5251263760808043, 0.014168949566985084, -0.07496818535288433, 0.7145916486076117, -3.4752911912482665, -1.018841409310305, -1.427445598193455, 2.908413097033151, 1.304995752314437, 0.6170016673540517, 1.868162423533912, 1.7476308933657503, 1.1446659650193711, 1.3909377946668358, 1.5982565485391256, 0.6495279843750664, 1.2636416893507734, 0.6683966550708224, 1.047622469860684, 0.5165141230762873, 0.8413421376613986, 1.572370609656174, 1.502491482293301, 1.0425916413508476, 1.3686863592112808, 1.7738171556494908, 0.8900752852214443, 1.014856533296381, 1.1332347191837757, 0.17894752854577564, 1.278056375993234, -0.30058872822066485, 2.4149836927324237, 1.339601609036446, 0.8297861431163488, 0.37467994523304793, 1.7471786854004023, 1.454137222418577, 1.8068308524592118, 1.552375158183832, 1.8263181457590665, 0.8290139809219107, 1.3602783236782738, 2.2181804447681333, 1.0608638466548108, 1.3222678220872672, 1.6556227504796304, 1.6654957169966358, 0.7511498549330112]

valid_com= ['Nn1c(=O)oc(=O)n2ncn2c1=O', 'O=C(CSCc1ccc(O)cc1)Nc1cccc(OC(F)(F)F)c1', 'C[NH+](C)Cc1ccc2ccccc2c1Cl', 'Clc1nc(-c2ccccc2F)nc2c1cnn2-c1ccc(Br)cc1', 'ClCc1cc(Br)ccc1O', 'Clc1cncc(C2CC2)c1', 'O=C(c1cc2ccccc2oc1=O)N1CC[C@@H](n2cncn2)C1', 'OCCC(=O)N1CCN(C(=O)c2cccs2)CC1', 'Oc1ccc([N+](=O)[O-])cc1/C=[NH+]/C(Cl)=C/Cl', 'O[C@@H]1CCCCC2(CCN([NH2+]Cc3ccc(OC(F)F)c(F)c3)CC2)C1', 'CCc1cccc(NC(=O)C#CCc2ccccc2)c1', 'Cc1ccc2nc(NC(=O)c3ccc(C(=O)[O-])cc3)sc2c1C', 'C=CCn1cc(NC(=O)c2cc(C(C)C)ncc2F)cn1', 'COCCCNC(=O)NCC[NH+]1[C@@H](C)CCC[C@@H]1C', 'CN(Cc1ccc([O-])cc1)C(=O)c1ccc(Cl)c(N2CCCCC2)c1', 'C[C@@H](OCC1CC1)C(=O)N1CC[C@@H](Oc2ccccc2F)C1', 'Clc1ccc(CN(C(=O)CCC2CC[NH2+]CC2)C(F)(F)F)cc1Cl', 'ClC(=O)N[C@H](Cc1cccs1)C(=O)[O-]', 'ClCc1nnc(COc2ncccc2F)s1', 'ClC[C@@H](CO)c1cccs1', 'ClC[C@H]1CCCC[C@@H]1c1ccccn1', 'Nc1ccc(C(=O)N2CCC3(CC2)c2ccccc2NC(=S)N3C)cc1', 'N=C1/C(=C/c2ccc(OCc3cc(C(=O)NCc4cc[nH+]cc4)ccc3)cc2)C(=O)SC1', 'N[C@@H]1Oc2ccc(C(=O)Nc3ccccn3)cc2C(=O)N1c1ccccc1Cl', 'N#C[C@@H](c1ccccc1)[C@H]1CCC[NH+](C2CN(C(=O)Cc3cnn(C4CC4)c3)C2)C1', 'NS(=O)(=O)[C@@H]1CCCN(C(=O)CSc2cccc([N+](=O)[O-])c2)C1', 'CCC(CC)C(=O)N[C@H](C)c1ccc(OC)cc1', 'CC(C)NC(=O)C(=O)Nc1cccc(SC(C)(C)C)c1', 'CC[S@](=O)CC(=O)N1CCN(C(=O)Nc2ccccc2-n2cncn2)CC1', 'CCc1ccc(NC(=O)N2CCN(c3cccc(C)c3)CC2)cc1[N+](=O)[O-]', 'CC1CCN(C(=O)c2cc(C(=O)N(Cc3ccccc3)c3ccccc3)ccc2F)CC1', 'CCOC(=O)N1CCC2(CC1)C[NH+]=C(N)N2c1ccccc1', 'CCNC(=O)[C@H]1C[C@H](c2ccccc2F)N(C(=O)c2ccc(F)c(F)c2F)C1', 'CC[C@@H]1CN(C(=O)NCCNC(C)C)CCO1', 'CCn1nc(CNC(=O)CC(C)C)c(=O)c(C)c1C', 'CCS(=O)(=O)Cc1ccccc1OC[C@@H]1Cc2ccccc2O1', 'OCc1ccc(N2CCC[C@H](c3nn(CC(F)F)c4ccccc34)C2)c(Cl)c1', 'OC1=Nc2ccccc2C(c2ccccc2)(c2ccc(Br)cc2)C1', 'OC[C@@H](NCc1ccc(OCCO)cc1)c1ccccc1', 'OCSc1nc2ccccc2cc1NCc1cccc(Br)c1', 'OC[C@H]1CCCC[NH+]1Cc1cccc(OCc2ccccc2)c1', 'OC[C@@]1(NC2CC2)COc2ccccc21', 'Clc1ccc2ccccc2c1Oc1ccccc1C[NH2+][C@@H]1CCS(=O)(=O)C1', 'Clc1ncnc(N2CC=C(c3ccc(OC(F)F)cc3F)CC2)c1[N+](=O)[O-]', 'Clc1nncnc1N[C@H]1CCc2ccccc21', 'ClCc1ccc(C(=O)N2CCC(Oc3ccc(F)cc3F)CC2)cc1', 'Clc1ccc(SCCCC(=O)N2CC[C@H](c3ccccc3)C2)[nH]1', 'ClC[C@H]([NH2+]Cc1cc(O)ccc1O)N1CCOCC1', 'ClC[C@H]1CO[C@H](N2CC[NH+](Cc3ccco3)C2)CC1', 'CNC(=O)c1cc(C(=O)N[C@H](C)c2cc(Cl)ccc2OC)ncn1', 'CN(Cc1ccc([N+](=O)[O-])cc1)C(=O)NCc1ncn(C)n1', 'CNc1cccc(OCC(=O)N2CCCC2)c1', 'CN=C1C(=O)N=C(N2CC(=O)N(Cc3cccnc3)C2=O)N=C(N)N1c1ccccc1', 'CNS(=O)(=O)c1ccc(=O)n(CCC(=O)N(C)Cc2ccc(Cl)cc2)c1', 'CN[C@H](c1cccs1)c1cccc(-n2cncn2)c1', 'O=[N+]([O-])c1ccc(N[C@H]2CCC[C@H](S(=O)(=O)CCc3ccccc3)C2)cc1', 'O=C(NCCNc1cccc2ccccc12)c1c[nH]c(=O)[nH]c1=O', 'Oc1ccccc1[C@@H]1CC[NH+]2CCCC[C@@H]12', 'Nc1cc([O-])nc([C@@H]2CCCN(c3ncnc4ccsc34)C2)n1', 'Clc1nnc([N+](=O)[O-])c(OC2CCCCC2)n1', 'Clc1nnnn1CCOc1ccc(Br)cc1', 'O=C(COC(=O)c1cnc(Br)cn1)Nc1cc(Br)ccc1Cl', 'O=[N+]([O-])c1ccc(CN2CCN(c3ncnc4scnc34)CC2)cc1', 'Cc1cc(F)ccc1NC(=O)[C@@H]1CCCN(C(=O)C(=O)NCc2ccc[nH]2)C1', 'Clc1ccsc1Sc1ccc(Cl)c(C(=O)[O-])c1', 'Clc1cc(Br)cnc1Oc1ccccc1[N+](=O)[O-]', 'Clc1ccc(OC[C@H]2CO2)cc1F', 'O[C@@H](c1ccc(Br)o1)C1CCN(S(=O)(=O)c2ccc(F)cc2)CC1', 'O[C@@H]1C[C@H](c2cccs2)C2(CCCCC2)N1[C@@H](Cn1cccn1)N1CCOCC1', 'N[C@@H](c1ccc(F)cc1)c1c(F)cccc1Cl', 'N[C@@H]1C[C@@H]([NH3+])CN1Cc1cccc(F)c1', 'C=CCNC(=O)C[NH+](Cc1cccs1)[C@@H]1CCS(=O)(=O)C1', 'Clc1nnns1', 'Clc1nnn(CC2CCCCC2)n1', 'Clc1nnnc(NCc2ccc(Cc3cc(F)ccc3F)s2)c1', 'Clc1nnnn1C1CCN(c2cccc[nH+]2)CC1', 'C[C@@H](NCC1CCC(C(N)=O)CC1)c1nccnc1C1CC1', 'O=C(C/c1c(F)cccc1F)[C@@H]1Cc2ccccc2S1', 'O=C(c1ccc(Cl)cc1)N1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1', 'O=C([O-])[C@@H](Cc1ccccc1)N1CCc2ccccc21', 'O=C([C@H]1CCCN1C(=O)c1cnc2ccccc2c1)N1CCN(c2ccccc2C(F)(F)F)CC1', 'O=C(/C=C/O)C(=O)NCc1ccccc1Br', 'Cc1ccccc1OCC(=O)N1CCO[C@@H](c2cccc(C(F)(F)F)c2)C1', 'Cc1nc(C(=O)N2CCC[C@@H]2C(=O)c2c(C)cc(C)cc2[N+](=O)[O-])c2ccccc2n1', 'Cc1sc2nc(NC(=O)C3=CN(c4ccccc4)CC3)sc2c1C', 'N#CC[C@]1(O)CCCN(C(=O)c2cn(Cc3ccccc3)n[nH+]2)C1', 'Oc1ccc(Cl)c(C(=O)N2CCCCCC2)c1', 'C[C@@H](CC(=O)Nc1ccc2c(c1)OCO2)CCC(C)(C)C#N', 'C[C@@H](c1ccc[nH]1)N(C)C(=O)[C@@H]1CC(=O)N(Cc2ccccc2F)C1', 'C[C@@H](Oc1ccc(Cl)cc1)C(=O)N1CCN(S(=O)(=O)c2ccccc2)CC1', 'C[C@@H](Nc1ncccn1)[C@@H]1CCCN(c2ncccc2C(C#N)=O)C1', 'C[C@@H]([C@@H](C)CNC(=O)C1CCC1)NC(=O)c1cccs1', 'C[C@@H]([NH2+][C@@H]1CC[C@@H]([NH+](C)C)C1)c1cccc(Cl)c1', 'ClC[C@@H]1C=C[C@@H](c2ccccn2)C1', 'OC[C@@H](C(=O)NCCCn1cccn1)Nc1ccccc1F', 'OC[C@@H]1CC2=C(C[C@H]1O)C(=O)O2', 'N=C1N2CCS(=O)(=O)C[C@@H]2[NH2+]Cc2ccccc21', 'ClC(c1ccc([N+](=O)[O-])cc1)[C@H]1CCOC2(CCC2)C1', 'ClC(=O)c1ccc(N[C@@H]2CSCCS2)cc1', 'ClC(F)(F)COCO[C@H]1CCOc2ccccc21', 'ClC([O-])=CCC[C@H](c1ccccc1)N1CCCCCC1', 'ClC(/N=C1/N=C[C@@H]2c3ccccc3C(=O)N12)[C@@H]1COc2ccccc21', 'COC(=O)[C@H]1CCCC(C)(C)[C@@H]1[NH3+]', 'COc1ccccc1C(=O)OCCC(=O)N1C[C@H](C)C[C@@H](C)C1', 'O=[N+]([O-])c1cccc(-c2nnc(CC[NH+]3CCCC3)o2)c1', 'C=CCNC(=O)c1ccc(C)c(OC)c1', 'C=C(C)C[NH+]1CCCC[C@@H]1c1nc(C2CC2)no1', 'C=C[C@@H](CC(C)C)[NH2+]C1CCN(C(=O)Nc2ccccc2)CC1', 'C=C1c2cc3ccccc3cc2C(=O)N1c1ccc(C)cc1', 'O[C@@H](COCc1ccc2c(c1)CCCCC2)C[NH+]1CCC[C@@H](C(NC2CC2)C(F)(F)F)C1', 'O[C@@H](Sc1nncn1Cc1ccccc1)Cc1nnc2ccccn21', 'O[C@@H](c1ccc2c(c1)CCC2)C(F)(F)F', 'O[C@@H]([C@@H]1CCC[NH+]1Cc1c(Cl)cccc1Cl)NCc1ccccc1', 'Cc1c([C@H](C)NC(=O)c2ccc3c(c2)OCCO3)cccc1[N+](=O)[O-]', 'Cc1ccc(S(=O)(=O)N2CCCC[C@@H]2C(=O)NOc2cncc(C(F)(F)F)n2)cc1', 'Cc1cn[nH]c1C(=O)N[C@H]1CC[C@@](CO)(C(C)(C)C)C1', 'Nc1[nH]nc(CN[C@@H]2CC[C@@H]([NH+](Cc3ccccc3)C(=O)NCCCC(=O)[O-])c2)c1Cl', 'Nc1cn[nH]c1C(=O)CCCCS(=O)(=O)c1ccc(F)cc1', 'Nc1nc(-c2ccc(Cc3cccs3)nc2)ccc1C(F)(F)F', 'COc1ccc(C[C@@]2(C)NC(=O)N(CC(=O)N(C)S(=O)(=O)Cc3ccco3)C2=O)cc1', 'N#CC1(SCc2ccco2)CCCCC1', 'N#Cc1ccc(CN2CCN(C(=O)COc3ccccc3F)CC2)nc1', 'N#C[C@@H](c1ccccc1)[C@H]1CCCN1C(=O)N[C@H](c1cccc(C)c1)c1ccccc1', 'N#C[C@H](c1ccccc1)[C@@H]1CCCN1Cc1ccc(F)c(F)c1', 'N#C/C(=C\\c1ccc(-c2cccc([N+](=O)[O-])c2)o1)C(=O)NCCc1ccccc1', 'N#C[C@@]1(Sc2ccccc2)NC(=O)C=C1c1ccccc1', 'O=[N+]([O-])c1cccc(CNC2CCSCC2)c1', 'O=[N+]([O-])c1ccc(CN2CCOc3ccc(Cl)cc32)c(Cl)c1', 'O=[N+]([O-])c1ccc2[nH]cc(-c3ccn[nH]3)c2c1', 'OCSCCOCCc1n[nH]c2cc(OC(F)F)ccc12', 'OCS(=O)(=O)c1ccccc1F', 'OCSc1nnc(N2CCC[C@@H]2[C@@H]2CCCC2)[nH]1', 'OCS1(=O)CCCCC1', 'OCS[C@@H]1CCCc2[nH]nc(-c3ccccc3)c(=O)c21', 'OCS[C@H]1CC[C@](NN=C2/C(=CCc3ccccc3)NN2Cc2ccccc2)N1', 'OCS/C=C/C1=NN2CCCC[C@@H]12', 'Clc1cc(C[NH+]2CCC[C@@H]2CO)ccc1O', 'Clc1cc(NCc2ccc(C[NH+]3CCCC3)cc2)ncn1', 'Clc1cc(F)cc(C[NH+]2CCC[C@H](c3nc(-c4ccco4)[nH]n3)C2)c1', 'Clc1cc(Cl)cnc1N/N=C1\\N=C(c2ccco2)S1', 'Clc1cc(Br)c(CN2CCN(c3cnc4cc(Cl)ccc4c3)CC2)cc1', 'Clc1cc(/C=C2\\C(=O)NC(=S)N(c3ccccc3)C2=O)ccc1Cl', 'N[C@@H](Cc1c[nH]c2ccccc12)C(=O)Nc1cccc(Cl)c1', 'N[C@@H](c1cccc(Br)c1O)[C@@H]1C[NH+]2CCN1CC2', 'N[C@@H](O)c1cccc(NC(=O)N[C@H](CSC(F)F)c2cccc(F)c2)c1', 'N[C@@H](Nc1nc2c(s1)C[NH+](C1CC1)CN(Cc1cccs1)C2)C1CC1', 'N[C@@H](Sc1ccccc1)c1ccccc1[N+](=O)[O-]', 'COc1ncnc(N)c1CN1CCN(c2ccc(F)c(F)c2)CC1', 'COc1ncccc1N(CC)Cn1nc(C)cc1C', 'COc1nnc(SCc2ccc(C(=O)[O-])cc2)n1CCOC(C)C', 'Cc1sc(-c2cccs2)cc1[C@H](C)NC(=O)Nc1ccc(C#N)cc1', 'Cc1snc(-c2ccco2)c1CNC(=O)/C=C/c1cccc(OC)c1', 'CC1=C(C(=O)Nc2ccccc2)c2ccccc2C(=O)N1C', 'CC1CCC([C@@H](C)NC(=O)Nc2ccc(Cl)cc2)CC1', 'CC1(C)[C@H]([NH2+]CNc2ccccc2)C(C)C1', 'CC12CCCC(CCCC2)CC1(Br)Br', 'C=CCCC(=O)Nc1ccc2c(SCC(F)F)cc(C)nc2c1', 'C=CCc1cccc(NC(=O)N2CCC(CNC(=O)C(C)C)CC2)c1', 'C=CCOc1ccc(/C=N/NC(=O)Cc2ccc(C)cc2)cc1', 'C=CCNC(=O)[C@H]1CCC[NH+](Cc2c[nH]nc2-c2ccc(C)o2)C1', 'C=CCn1cc(C(=O)NNC(=O)N2CCN(c3ccc(Cl)cc3)CC2)c2ccccc21', 'C=CC#CCc1cccc(NC(=O)N2CCC(NC(=O)c3ccccc3)CC2)c1', 'C=CC[NH+](CC(=O)[O-])Cc1ccc(OC)c(OC)c1', 'Clc1nnc(SCC2CC[NH2+]CC2)nn1', 'Clc1nncn1CCN[C@H](c1ccccc1)c1ccccc1F', 'Clc1nncc(Cl)n1', 'COCCn1c(S[C@H](C(N)=O)C(=O)Nc2ccc(Cl)cc2)c(-c2ccccc2)c(C)n1', 'COC(=O)C[C@H]1CCC[C@H]1NC(=O)NCc1cnn(C)c1', 'COCc1cc(NC(=O)N[C@H]2CC(=O)N(C3CCCCC3)C2)n(C)n1', 'COC[C@@H]([NH2+]CC1CCCC1)C1CC1', 'COC[C@H](O)C[C@@H](C)C(=O)N1CCN(c2ccc(F)cc2)CC1', 'COC[C@](C)(O)CNC(=O)N(C)[C@@H](C)C(C)(C)C', 'O=C(c1cnn2cc[nH]c12)OCc1cnn(-c2ccc(Cl)cc2)c1', 'O=C([C@H](O)c1ccc(Cl)c(Cl)c1)N1CCN(C(=O)c2cccc(F)c2)CC1', 'O=C([C@H]1Cc2cc(Cl)ccc2O1)N1CCC(O)CC1', 'Oc1cc(/C=N/NC(=O)C[C@@H](c2cccs2)N2CCOCC2)nc2ccccc12', 'O=[N+]([O-])c1ccc2c(c1)N(Cc1cc(C(F)(F)F)cs1)CC2', 'Oc1cc(Cl)cnc1SCc1ccc(F)cc1', 'Oc1ccc(Cn2cc([C@@H]3CC[NH2+]C3)nn2)cc1I', 'Clc1nncnc1C[C@H]1CCC[NH+](CCc2c[nH]c3ccccc23)C1', 'Clc1nncn1/c1ccccc1', 'NS(=O)(=O)c1cc(NC(=O)COc2ccc(Br)cc2)ccc1O', 'NS(=O)(=O)c1ccc(CNS(=O)(=O)c2cc(Br)ccc2Cl)cc1', 'N=CC[C@H]1C(=O)NC(=O)C[C@@]1(O)CSc1ccccc1', 'N=Cc1cccn1CC(=O)N1C[C@H](C)OCC1(C)C', 'N=C1\\C[NH+](CCCCN2CCCCC2)[C@H](C)C(=O)N1c1ccccc1', 'O=[N+]([O-])c1c(Cl)nc2[nH]ncc2c1Cl', 'O=[N+]([O-])c1cccc(NC2CC2)c1', 'O=C([O-])c1ccc(-n2cccn2)cc1C(=O)NC[C@H]1Cc2ccccc2O1', 'O=C(/C=C/c1cccs1)NNC(=O)Oc1ccccc1C(F)(F)F', 'O=C(/N=C/C(=O)Nc1cc(C(F)(F)F)ccc1Cl)c1c[nH]c2ccccc12', 'O=[N+]([O-])c1cc(CNc2ccc(Oc3ccccc3)nn2)cc2c1OCCO2', 'O=[N+]([O-])c1ccc(S[C@@H]2CCCS[C@@H]2c2cccs2)cc1', 'O=[N+]([O-])c1ccn(C2CC[NH2+]CC2)c1', 'Clc1ccc(C[NH+](Cc2ccccc2)Cc2ccccc2F)cc1', 'Clc1cccc(CC[NH+]2CCCCCCC2)c1', 'C[C@@H](OC(=O)[C@@H]1CCCO[C@H]1C)c1ccc2c(c1)OCCCO2', 'C[C@@H](O)C[C@@](C)(O)CNC[C@@H]1CCC[NH+]1C', 'C[C@@H](Oc1ccccc1)C(=O)Nc1ccc([N+](=O)[O-])c([N+](=O)[O-])c1', 'NS(=O)(=O)c1ccc(S(=O)(=O)C[C@H](O)c2ccccc2)cc1', 'NS(=O)(=O)c1ccc(NC(=O)N2CCC(CS(=O)(=O)c3ccc(F)cc3)C(=O)C2)cc1', 'NS(=O)(=O)c1ccc(CNC(=O)c2cccnc2)cc1', 'Nc1c(Cl)cccc1NC(=O)[C@@H]1CC(=O)N(CCc2c[nH]c3ccccc23)C1', 'Nc1cncc(/[NH+]=C/c2ccc(C#N)cc2)n1', 'NS(=O)(=O)c1cccc(NCC[NH+]2CCCCC2)c1', 'NS(=O)(=O)N1CCC[C@H]1c1cc(F)cc(F)c1', 'NS(=O)(=O)c1ccc(C(=O)N2CCN(c3cc(-c4cccc(F)c4)on3)CC2)cc1', 'Nc1noc2cc(CCSc3ccc(O)cc3)ccc12', 'Nc1n[nH]c2ccccc12', 'Nc1ncccc1Oc1ccccc1OCc1ccccc1Cl', 'Nc1nn(-c2ccc(N3CCCCC3)cc2)c2c1CCC2', 'N#Cc1ccccc1O[C@@H](C)C(=O)N1CCOC[C@@H]1C1([NH+](C)C)CCCCC1', 'NS(=O)(=O)N[C@@H](c1ccccc1)c1ccc(Cl)cc1', 'NS(=O)(=O)/C=C(\\c1ccccc1)N1CCN(Cc2ccc3c(c2)OCCO3)C1', 'NS(=O)(=O)CCNC(=O)[C@@H](C#N)c1ccc2c(c1)CCC2', 'Nc1[nH]ncc1COC(=O)[C@H]1C[C@H]1c1ccc(Cl)cc1', 'NS(=O)(=O)NCc1cc(C2CC2)ccc1[O-]', 'NS(=O)(=O)N(Cc1ccc(Cl)cc1)C1CCSCC1', 'NS(=O)(=O)Nc1cccc(NC(=O)c2cccc(F)c2)c1', 'NS(=O)(=O)N1CCC2(CC1)Oc1ccc(F)cc1C(c1ccccc1)(CCC[NH+]1CCOCC1)CO2', 'NS(=O)(=O)N[C@@H]1CCN(S(=O)(=O)N(C)Cc2cccc(F)c2)C1', 'NS(=O)(=O)N[C@H]1CC[NH+](Cc2ccc(F)cc2)C1', 'Oc1cc(C(=O)N[C@@H]2COc3ccc(Cl)cc32)cs1', 'Oc1cc(Oc2ccc(CS(=O)(=O)Cc3ccc(Cl)cc3)cc2)ccc1F', 'Oc1cc(F)ccc1N[C@@H]1CCO[C@]2(CCSC2)C1', 'Oc1cc(-c2noc(C3CC3)n2)ccc1SCC(=O)Nc1cccc(Cl)c1', 'Oc1cc(Br)ccc1Nc1cnccc1C[NH+]1CCSCC1', 'ClCc1ccccc1O/N=C1/N=C(c2ccccc2)SC1=O', 'ClCc1ncnc(N2CCOCC2)c1-c1ccc(F)cc1', 'ClC[C@@H]1CCC[C@@H]1COCCn1cccn1', 'ClC[C@@H]1OCCN(C(=O)Cc2ccccc2)C1', 'ClCc1noc(-c2ccccc2)n1', 'ClCc1n[nH]c2cc(-c3cc(F)cc(F)c3F)oc12', 'ClCc1nc[nH]n1', 'ClCc1nnc(Oc2cccc([N+](=O)[O-])c2)n1C1CC1', 'ClC[C@@H]1OCCCN1C[C@@H](O)c1ccc2c(c1)CCCC2', 'ClC[C@@H]1O[C@H]2OCCN2[C@@H](c2cccnc2)O1', 'ClC[C@@H]1Oc2ccccc2Oc2ccccc21', 'Cc1noc(C)c1CCC(=O)N(C)Cc1ccccc1', 'Cc1nc(C(C)C)c2nc(Nc3ccc(Cl)cc3)sc2n1', 'Cc1nnnn1CC(=O)N[C@@H](C)c1nc(C)cs1', 'CNS(=O)(=O)c1ccccc1NC(=O)CCN1Cc2ccccc2OC1', 'CN(CC1CCC([NH+]2CCCCC2)CC1)C(=O)NCCCn1cncn1', 'CN(c1cccc(Cl)c1)S(=O)(=O)c1c(C(N)=O)cnc2ccccc2c1=O', 'O=C(CCC(=O)N1CC[C@@H](CCCO)C1)Nc1cccc(OCC(F)(F)F)c1', 'O=C(Cc1cccc(Cl)c1)NCc1cccc(-c2ncnn2C2CC2)c1', 'O=C(C1CCC(Nc2ccccc2Cl)CC1)c1ccc(Cl)cc1', 'O=C(COc1ccccc1Cl)N[C@H](Cn1cc[nH+]c1)c1ccccc1', 'O=C(CN1CNc2[nH]c[nH+]c2[C@@H]1c1ccccc1)NC[C@H](O)c1ccccc1', 'O=C(CSc1nc2ccccc2c(=O)n1Cc1ccccc1)NNC(=O)c1ccccc1F', 'O=C(C[NH+]1CCN(c2ccccc2)CC1)N1CCN(Cc2ccc(F)cc2)CC1', 'O=C(c1cccc(Br)c1)Oc1ccc(Cl)cc1', 'O=C([O-])C1=NO[C@@H]2NC(=O)N(c3ccccc3)/C1=N/OCCC2', 'O=C([O-])c1cnc(=S)n(-c2nccc(Oc3cccnc3)n2)c1', 'O=C([O-])[C@@H]1CCCN(C(=O)Cc2c[nH]c3ccccc23)C1', 'O=C([O-])[C@H](Cc1ccc(F)cc1)NC=C1C[NH2+]CCc2ccccc21', 'O=C([O-])/C=C/c1ccccc1OCC1CC1', 'Nc1c(C(=O)N[C@H](CO)c2ccc(Cl)cc2)cnn1-c1ccccc1', 'Nc1c(Oc2ccc(F)cc2)nnn1-c1ccc(F)cc1', 'Nc1c(F)cccc1N1CC[NH2+]CC1', 'Nc1c(-c2cccc(Cl)c2)ncnc1Nc1cc(F)ccc1F', 'Nc1c(Cl)cccc1N[C@H]1CCN(Cc2ccccc2Cl)C1=O', 'Nc1c(/C=N/O[C@H]2CCCc3ccccc32)ncnc1NCc1ccc[nH]1', 'N=C1CSc2ccc(C(=O)NC3CCCC3)cc2N1', 'N=C1\\C(=C\\c2ccc(C#CC[NH+]=C/c3ccccc3)cc2)NC[C@@]1(O)Cc1ccccc1', 'N=C1N(C[C@H]([NH2+]C)c2ccccc2F)C(=O)NC([S-])=NC1', 'N=C1[C@H]2CCCCN1Cc1cc(F)ccc12', 'N=C1/C(=C/c2c(OCc3ccccc3)sc3ccccc23)C(=O)N1c1ccccc1', 'Cc1sn(-c2ccccc2)c(=O)c1NC(=O)c1cc[n+]([O-])cc1', 'Cc1snc(C)c1S(=O)(=O)N[C@@H](CO)c1cccc(Cl)c1', 'Cc1snnc1CN1CCC(C(=O)NC2CCCCC2)CC1', 'CCc1cccc(Oc2ccccc2NC(=O)Cc2ccccc2)c1', 'CCCCN(Cc1ccco1)C(=O)Nc1ccc(Cl)cn1', 'CCC(=O)N1CCC2(CC1)OC(C(=O)N1CCC(C)CC1)CC2', 'CCCNC(=O)CC1CCN(C(=O)c2cccc3[nH]cnc23)CC1', 'CCC[C@@H](C(=O)N(C)[C@H](C)C(C)C)c1ccc(Cl)cc1', 'CCCn1nnnc1COc1cccc(Br)c1C', 'CCCS(=O)(=O)N1CCN(C(=O)[C@H]2CCCC[C@H]2[NH3+])CC1', 'CCC[C@H]1CCC[C@@H]1C(=O)NCCCC[NH+](C)C', 'CCC[C@]1(C)CCN(c2ccc([N+](=O)[O-])c([N+](=O)[O-])n2)C1', 'CCC[NH2+]Cc1cc(S(C)(=O)=O)ccc1[N+](=O)[O-]', 'O[C@@H]1Cc2ccccc2[C@@H]1Cc1c(F)cccc1F', 'O[C@@H]1c2ccccc2C(=O)N1CCOc1ccc(Br)cc1', 'O[C@@H]1[C@@H]([C@@H](O)C[C@@H]2CCCOCC2)c2cc3ccccc3nc21', 'Clc1nnn(-c2cccc(Cl)c2)c1Cc1ccccc1Cl', 'Clc1nnn(C[C@@H]2CCc3ccccc32)c1Cl', 'Clc1nnnc(N2CCC[C@@H](Cc3ccccc3)C2)n1', 'Clc1nncc(-c2ccc(Cl)c(Cl)c2)n1', 'Clc1nncc(CO/N=C/c2ccccc2Cl)n1', 'Clc1nncc(OCCCO[C@H]2COc3ccccc3O2)c1[N+](=O)[O-]', 'Clc1nncc(NCCc2nc(C3CC3)no2)c1O', 'Clc1nncc(F)c1-c1ccc(F)cc1', 'Clc1nncc(-n2cccc2)c1-c1cc(F)ccc1F', 'Clc1nncc(Sc2ccc(Cl)cc2)c1NC1CCCCC1', 'Clc1nncc(Cl)c1I', 'Clc1ccsc1Cn1c(=O)n(Cc2ccco2)c2ncccc21', 'Clc1ccsc1SC[C@H]1C[C@@]12OCCC[NH2+]C2', 'Oc1ccc(C(=O)c2sc(N3CCOCC3)cc2C(=O)[O-])cc1', 'Oc1cccc(F)c1SCC(=O)N1CSC[C@@H]1c1ccccc1', 'OCc1ccc(-c2csc(-c3ccccn3)n2)cc1', 'OC[C@@]1(OC2CC2)CCC[C@@H]1c1ccccc1F', 'Oc1ccc([N+](=O)[O-])c(COc2ccc(C(F)(F)F)cc2)c1', 'Oc1ccc(Oc2ccc(C(=O)[O-])cc2)cc1', 'Oc1ccc(F)cc1[N+](=O)[O-]', 'Oc1ccc([C@H]2CCCC(C#N)(C3CCOCC3)c3ccccc3N2)cc1', 'Oc1ccc(/C=N/Nc2ncnc(Nc3ccccc3)c2N2CCCC2)c2c1CCCC2', 'Cc1noc(C)c1S(=O)(=O)/N=C(\\[O-])[C@]1(C)CC1(Cl)Cl', 'Cc1nonc1C(=O)N(CC1CCOCC1)[C@H]1CCS(=O)(=O)C1', 'OC1=Nc2ccccc2C[C@@H](c2ccccc2)C1', 'OC1CC[NH2+]CC1', 'OC1(c2ccccc2)CCN(C(=O)c2csc(=O)[nH]2)CC1', 'Clc1nnnc(CN2CCC[C@@H]2c2ccccc2)n1', 'Clc1nnnc(Oc2cncc([N+](=O)[O-])c2)n1', 'Clc1nnnc(NC2CCC(Oc3ccccc3)CC2)c1C1CC1', 'Clc1nnnc(-c2ccccc2)n1', 'Clc1nnnc(SCc2cc(F)ccc2F)n1', 'Clc1nnnc(Cl)c1Cc1ccc(Cl)cc1', 'Clc1nnnc(Br)c1Br', 'Clc1ccc(CCC[NH2+]Cc2ccc[nH]2)cc1Cl', 'Clc1ccc([C@@H](CNCc2ccccc2)N2CCCCC2)c(Br)c1', 'Clc1ccc(-c2nnc(CSC[C@@H]3CCCO3)s2)cc1', 'Clc1ccc(Br)c([C@H]2CCC[NH+]2Cc2nnco2)c1', 'C=C1NC(=O)N([C@@H](C)c2ccccc2C)C(=O)[C@H]1/C=C/c1ccc2c(c1)OCO2', 'C=C1[C@H](C(=O)Nc2cc(Cl)c(C)cc2C)S[C@@H](C)C[C@H]1C', 'C=C1c2ccc(OC)c(C(=O)NCc3ccccc3)c2[C@H]1c1ccc(SC)cc1', 'N[C@@H]1c2ccccc2CCC1=O', 'N[C@@H]1NC(=O)/C(=C/c2ccco2)S1', 'N[C@@H]1[C@H](C(=O)OCCSc2ccccc2)Cc2ccccc21', 'N[C@@H]1[NH2+]CC[NH+]([C@H](Cc2ccccc2)c2ccccc2)C1', 'ClC[C@H](CS(=O)(=O)c1ccc(Br)c(F)c1)NC(=O)c1ccc2c(c1)OCCO2', 'ClC[C@H](c1ccc(Br)s1)N1CCC2(CC1)COc1ccccc12', 'ClC[C@H](O)C1CC[NH+](C2CC2)CC1', 'ClC[C@H](Br)c1c(Cl)cccc1Cl', 'ClC[C@H]([NH2+]Cc1ccccc1Br)c1ccsc1', 'CNC(=O)c1cccc([C@@H]2CCCCN2C(=O)c2cc(F)ccc2)c1', 'CN[C@H](CO)c1cccc([N+](=O)[O-])c1', 'CN[C@H]1CC[C@@H]([NH+]2CCC(Cc3nc(C4CC4)no3)CC2)C1', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1NCCN1C(=O)N[C@](CC2CC2)C1', 'O=[N+]([O-])c1c(O)sc2c1CCN(Cc1cc[nH]n1)C2', 'O=[N+]([O-])c1c(F)cccc1N1CC[C@@H](C[NH+](Cc2ccccc2)C2CC2)C1', 'O=[N+]([O-])c1c(-c2ccccc2)n(-c2ccccc2F)nc1[O-]', 'O=[N+]([O-])c1c(Cl)cccc1C(=O)Nc1ncc[nH]1', 'O=[N+]([O-])c1c([O-])n(-c2ccccc2)c2ccccc12', 'O=[N+]([O-])c1c(Br)cccc1C(F)(F)F', 'Cc1c([N+](=O)[O-])cccc1N1C(=O)[C@H]2SCC[C@@H]12', 'Cc1c(C(=O)N[C@H](C(=O)N2CCCC2)nncc1C(=O)N1CCCCC1)c1ccccc1', 'Cc1c([C@@H](C)CCCCO)c(=O)n2c(-c3ccc(Br)cc3)n[nH]c2nn1C', 'Cc1c(-c2cncc(NC3CCCC3)c2)oc2ccccc12', 'Cc1c(Br)cccc1NC(=O)NCC(C)C', 'O[C@@H]1c2ccccc2C(=O)N1Cc1cccs1', 'O[C@@H]1CCCCCC[C@H]1[NH2+]C1CCCC1', 'O[C@@H]1C(C(=O)OCc2ccccc2)c2ccccc2S1', 'O[C@@H]1Cc2ccccc2CN1C(=O)COCc1ccccc1', 'O[C@@H]1COCC[C@@H]1[NH2+][C@@H]1CCOC2(CCC2)C1', 'O[C@@H]1C[C@@H](c2cccs2)N(c2ccc(Cl)cc2)C(=O)O1', 'O[C@@H]1CSc2ccccc2O1', 'O[C@@H]1C[C@H]1C(=O)Nc1ccc(Cl)s1', 'O[C@@H]1C[C@@](Oc2ccccc2)c2ccccc21', 'Nc1nc(CNCc2ccc[nH]2)nc2ccc(F)cc12', 'Nc1ncc(CC[NH2+]C[C@H]2CCOC2)[nH+]c1N', 'Nc1nc2ccccc2n1CCOCc1ccccn1', 'Clc1nnc(CSc2ccc(N3CCCC3)cc2)nc1N1CCCC1', 'Clc1nnc([S-])n1CCOC(=O)c1ccc(Cl)c(F)c1', 'Clc1nnc(NCc2cccc(F)c2)n1-c1ccccc1', 'Clc1nnc(-c2ccccc2)nc1CS[C@@H]1CCCc2ccccc21', 'Clc1nnc(SCc2ccc(F)cc2F)s1', 'Clc1nnc(Cl)nc1C1CC1', 'Clc1nnc(Br)cc1Oc1ncccn1', 'Clc1nnc(/C=C/CCO)c([O-])n1', 'N#CCCC[NH+](CCCBr)CC(F)(F)F', 'N#CC(/N=C/c1ccc([N+](=O)[O-])cc1)=C/C(=O)Nc1nccs1', 'N#CCc1ccc(CNC(=O)Nc2nc(C3CC3)n(-c3ccccc3)n2)cc1', 'N#CC1=C([O-])CC2=C(C(=O)N(Cc3cccc4ccccc34)C2)[C@@H]1c1ccccc1Cl', 'N#CCOc1ccc(/C=N/c2c[nH]cn2)cc1', 'N#CCNC(=O)c1cc(C(=O)N2C[C@H]3CCCC[C@@H]3C2)nn1-c1ccccc1', 'N#CCSc1ccccc1Cn1c(=O)[nH]c2ccccc21', 'Clc1ccsc1CN1CCN(C(=O)NC2CCCCCC2)CC1', 'Clc1ccsc1NC(=S)Nc1ccc2c3c(cccc13)CC2', 'Clc1ccsc1F', 'Clc1ccsc1[C@@H]1CC(C(=O)N2CCN(CCO)CC2)=NO1', 'Clc1ccsc1-c1nnc(C[C@@H]2C[C@H]2c2ccccc2F)o1', 'Clc1ccsc1SCc1ccc(Cl)cc1Cl', 'Clc1ccsc1[C@H]1CCC[NH+](Cc2ccco2)C1', 'Clc1ccsc1/C=N/NC1=NS(=O)(=O)c2ccccc21', 'C[C@@H](c1ccc([S@](C)=O)cc1)N(C)C(=O)Nc1ccc(C(N)=O)c(Cl)c1', 'COc1nn[nH]c1C(=O)C[C@H]1CCCN(c2ncccc2Br)C1', 'COc1nn(C)cc1CNC(=O)N(CCS(C)(=O)=O)c1ccccc1', 'COc1nnnc(Nc2ccc(OC)cc2)c1C', 'C[C@@H]([C@@H](C)O)[NH+]1CCN(C(=O)Cc2csc(C(C)C)n2)CC1', 'NS(=O)(=O)/C=C/c1ccc(F)cc1', 'NS(=O)(=O)/N=C(\\C=C1CCCC1)Nc1ccc2[nH]c(=O)[nH]c(=O)c2c1', 'Nc1[nH]ncc1CN(c1ccccc1)S(=O)(=O)/N=C/c1ccc(Br)cc1', 'Nc1[nH]nc(-c2ccco2)c1N', 'Nc1[nH]ncc1C(=O)Nc1cc2c(cc1Cl)c(=O)[nH]n2C', 'Nc1[nH]nc2c1[C@@H]1CCC[C@H]1C2', 'NS(=O)(=O)CCNC(=O)[C@@H]1OCC(=O)N(C)[C@H]1c1ccc(Cl)cc1', 'NS(=O)(=O)Cc1ccc([N+](=O)[O-])cc1', 'NS(=O)(=O)C[C@@H](Cc1cn2ccccc2c1)c1ccccc1', 'NS(=O)(=O)C[C@H](O)C1CCn2cccc2C1', 'COc1nc([O-])cc(C(=O)NCC[C@H](O)c2ccccc2C)c1N', 'COc1ncnc(Nc2cc(C)cc(C)c2)c1C', 'COc1ncc(CN2CCN(C(=O)c3cnn(C(C)(C)C)c3)CC2)cc1Br', 'COc1nc2ccc(Cl)cc2cc1C(=O)NCC(=O)NC1CCCC1', 'CNS(=O)(=O)c1ccc([N+](=O)[O-])cc1N', 'CNC(=O)NCC(=O)Nc1ccc(N(Cc2ccccc2)C(C)C)nc1', 'CNS(=O)(=O)[C@H]1CC[NH2+]C[C@@H]1C(C)C', 'Oc1cc(OCCn2cccc2Cl)c2ccccc2n1', 'Oc1cc(O)ccc1OCC(=O)NCC(=O)c1ccccc1O', 'Oc1cc(O[C@@H]2CCCc3ccccc32)c2ccccc2c1[O-]', 'Oc1cc(-n2cccn2)ccc1NC1=NC=CN=C1c1ccccc1Cl', 'Oc1cccc(C[C@@H]2CCCN2C(=O)c2ccnc(NC3CCCCC3)c2)c1', 'Oc1ccccc1CN[C@H](CO)c1cc([N+](=O)c2cccnc2)ccc1F', 'Oc1cccc2ccccc12', 'Oc1cc(CC[C@@H]2CCC[NH2+]C2)ncn1', 'Oc1cc(C(=O)N2CC=C(CCn3ccnn3)CC2)ccc1O', 'Oc1cc(C=O)ccc1OCC(F)(F)F', 'Oc1cc(COCc2ccncc2)ccc1O', 'Oc1cc(Cn2cnc3ccccc32)c2[nH]ccc2n1', 'Oc1cc(C[NH+]2CCSC[C@@H]2C2CCCC2)cs1', 'Oc1cc(C[NH2+]Cc2ccc(F)cc2)ccc1Oc1ccccc1', 'C=C[C@@H](C(=O)OC(C)(C)C)N1C(=O)C(=O)N(C)C1=O', 'C=C[C@@H]1CC[C@](C#N)([C@@]2(O)CCC(C)(C)[C@@H]2O)[C@@H]1O', 'O=[N+]([O-])c1cc(C(=O)Nc2ccc(Cl)cc2)ccc1N/N=C/[C@@H]1C=CC=C1Cl', 'O=[N+]([O-])c1cc(OCCc2ccccc2)ccc1N1CCCCC1', 'O=[N+]([O-])c1cc(N2CCN([C@@H]3Cc4ccccc4O3)CC2)ccc1Cl', 'O=[N+]([O-])c1cc(F)c(OC[C@@H]2CCCCO2)cc1Br', 'O=[N+]([O-])c1cc(S(=O)(=O)Nc2ccc(Cl)c(Cl)c2)cc(Cl)c1Cl', 'O=[N+]([O-])c1cc(Cl)nc([N-]SCc2ccc(OCc3ccccc3)cc2)c1', 'O=[N+]([O-])c1cc([C@H]2CC(=O)N(c3ccc(Cl)cc3)C2=O)ccc1[O-]', 'O=[N+]([O-])c1cc(Br)ccc1Sc1ccc(F)cc1O', 'O=C(c1c[nH]c(C(=O)N2CCC[C@H](c3nc4ccccc4[nH]3)C2)n1)N1CCCC1', 'O=C(c1cccc(C2CCOCC2)c1)N1CCC(/C(Cl)c2ccccc2)CC1', 'O=C(c1cnncc1Cl)N1CC=C(c2ccc(F)cc2)CC1', 'O=C(/C=C[C@@H]1CCCS1(=O)=O)c1cccc2ccccc12', 'O=C(/C(c1ccccc1)C1=CCCCC1)[C@@H]1CC=CCC1', 'O=C([C@H](CNc1ccc(C(F)(F)F)cc1)c1cccs1)NNC(=O)c1ccc2c(c1)OCO2', 'O=C([C@H](c1ccco1)N1CCC(c2[nH]cc[nH+]2)CC1)c1ccccc1', 'O=C([C@H](NS(=O)(=O)c1ccc(Cl)cc1)c1ccccc1)N1CCC[C@H](CO)C1', 'O=C([C@H](Sc1ccccc1)c1ccc(F)c(Cl)c1)NC1CCOCC1', 'O=C([C@H](Cl)O)c1ccc(Cl)cc1Cl', 'OC[C@H]([NH2+]Cc1ccccc1)C1CC1', 'OC[C@H]1NC(=S)c2cccnc2C1=C(c1cccs1)N1CCCC[C@H]1Cc1ccccc1', 'N[C@@H]1[C@H](C2CC2)N=C(NCc2cc(Cl)ccc2-n2ccnc2)C1', 'N[C@@H]1[C@H]2CCCC[C@H]2[NH+]1Cc1ccncc1', 'Oc1cccc2ccccc12', 'Oc1cccc2ncccc12', 'Oc1cccc(C(=O)C2=CC(CCOc3ccc4c(c2)OCCO4)C3)c1', 'Oc1cccc(Oc2cnnc(Nc3ccccc3)n2)c1', 'Oc1cccc(NCC(=O)N2CCC[C@@H]2c2ccc(Cl)c(Cl)c2)c1', 'Oc1cccc(F)c1C[NH+]1CCC(OCc2ccccc2F)CC1', 'Oc1cccc(-c2cccc(NC(=O)c3ccc(F)c(S(=O)(=O)N4CCCC4)c3)c2)c1', 'Oc1cccc(Sc2nnnn2-c2c(C(=O)[O-])cnn2-c2ccc(Cl)cc2)c1', 'Oc1cccc([C@H](O)Cc2nc3ccccc3s2)c1', 'Oc1cccc(Br)c1C(F)(F)F', 'O=[N+]([O-])c1c([O-])c(COc2cccnc2)c2ccccc2[n+]1[O-]', 'O=C(c1cc(-c2ccccc2)n[nH]1)N1CCN(S(=O)(=O)c2ccccc2Cl)CC1', 'O=C(c1ccccc1Sc1ccccn1)N1CCN(c2nccs2)CC1', 'O=C(c1ccsc1Cl)Oc1cccc2c[nH]cc12', 'O=[N+]([O-])c1c([N+](=O)[O-])c2c([nH]1)C[C@@H](C(=O)N1CCCCCC1)CCC2', 'O=C(/N=C1\\CC[NH+]=C1C1OCCO1)c1cnc(Cl)cc1Br', 'COc1nnn(CC2CCCC2)c1[O-]', 'COc1nnnc(N2CCc3nn(-c4ccc(F)cc4)n(C)c3c2C(=O)OC)c1C', 'COc1nnnn1-c1ccc2nnnn2c1', 'C[C@@H](CCc1ccc(Cl)cc1)c1nnc(NC(=O)C(=O)NC2CC2)s1', 'C[C@@H](C(=O)Nc1ccc(N2CCCCC2=O)cc1)n1cc(-c2ccc3ccccc3c2)nn1', 'C[C@@H](CO)C[NH2+]CCOC[C@H]1CN(C)CCO1', 'C[C@@H](CNC(=O)c1cccc(Br)c1)NC(=O)c1ccccc1', 'C[C@@H](C[C@@H]1CCCC[NH2+]1)Nc1ccccc1Br', 'C[C@@H](Cn1c(C[NH+](C)CCCN2CCOCC2)noc1=O)c1ccccc1', 'C[C@@H](C[NH+]1CCN(CC(=O)N2CCCCC2)CC1)c1ccc(Cl)cc1', 'C[C@@H](C[NH2+][C@@H]1CCN(c2ccccc2F)C1)N1CCOCC1', 'Oc1cc(-n2cccc2)cc(Cn2ccc3ccc(Cl)cc32)c1', 'Oc1cc(Br)c([C@H]2CCCN2CC(=O)NCc2ccco2)cc1F', 'Oc1ccc(/C=C2/SC(=S)N(c3cccc4ccccc34)C2=O)cc1', 'Oc1ccc(/N=C/c2cccc(C(F)(F)F)c2)nc1', 'Oc1cc(F)cc(CNCC2(N3CCOCC3)CC2)c1', 'Oc1cc(-n2cccc2)ccc1NC1CC[NH+](Cc2ccco2)CC1', 'Oc1cc(-n2nnc(-c3ccc(Cl)cc3)n2)ccc1N1CCCCC1', 'OCc1nc(Cl)cc(Oc2ccc(Cl)cc2Cl)c1', 'OCc1n[nH]c(Nc2cccc(-c3ccccc3)c2)c1C1CC1', 'OCc1nc(Cc2ccccc2)no1', 'OCc1nnnn1-c1ccccc1', 'OC[C@@]1([NH2+]Cc2cccc(Br)c2)CCCC1(C)C', 'Oc1ccc(F)cc1NC(=O)c1ccc([N+](=O)[O-])cc1O', 'Oc1ccc([N+](=O)[O-])c(Cl)c1Cl', 'Oc1ccc(O[C@H]2CC[NH2+]C[C@@H]2CC2CC2)c(Cl)c1', 'Oc1ccc(Oc2cc3occc3nc2C(=O)N[C@H]2CC=CCCC2)cc1', 'Oc1ccc(O[C@@H]2CC[NH2+]C[C@H]2Cc2ccccc2)cc1', 'Nc1n[nH]c(-c2cccc(Br)c2)c1[O-]', 'Nc1n[nH]nc1COC(=O)[C@@H](C)N1CCCCCC1=O', 'ClCc1csc(-c2cc(Cl)ccc2NCc2ccccc2)n1', 'ClCc1cccc(CO)c1I', 'ClCc1cncc(COc2ccc(C(=O)[O-])cc2F)n1', 'ClCc1csc(-c2ccccc2)n1', 'ClCc1cc(F)cnc1Oc1ccccc1', 'ClCc1cccc(OCc2cc(-c3ccccc3)n[nH]2)c1', 'ClCc1ccnc(-c2noc(-c3ccccc3)n2)n1', 'ClCc1cn(Cc2ccc(F)cc2)nc1S[C@@H]1CCSC1', 'ClCc1cnc(COCCCOC2CCCCCC2)s1', 'ClCc1cnn(-c2cccc(Cl)c2)c1Cl', 'ClCc1cn[nH]c1-c1ccccc1C(=O)Nc1ccc([N+](=O)[O-])cc1Cl', 'ClC[C@@H]1CCC[NH+]1Cc1ccccc1O', 'ClC[C@@H]1C(=O)N2CCCNC2=C1C(=O)c1cccc(F)c1', 'ClC[C@@H]1COC[C@@H]1C1CCC1', 'ClC[C@@H]1CN(Cc2cc(F)ccc2F)CC1', 'ClC[C@@H]1C[C@@H]1c1cccc(Br)c1', 'ClC[C@@H]1C[C@H]1c1cccc(Cl)c1', 'OC[C@@H](Cc1cc(Cl)cc(Cl)c1)Nc1cc(OCc2ccccc2)ncn1', 'OC[C@@H](c1ccccc1)N1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1', 'OC[C@@H](O)C[NH+]1CCC([C@H](O)c2ccc(Br)cc2)CC1', 'OC[C@@H](NS(=O)(=O)c1ccc(Cl)cc1Cl)c1ccc(Cl)cc1', 'OC[C@@H](Sc1nncs1)C(=O)Nc1cccc2ccccc12', 'OC[C@@H](Br)C1CCC(C(F)(F)F)CC1', 'OC[C@@H]([NH2+]Cc1ccncc1)Cc1ccccc1', 'ClCc1noc(Cn2cnnn2)c1N1CCCN(c2ccccc2)CC1', 'ClCc1nonc1COc1ccccc1', 'N=Cc1cncc(CN2CCCC[C@@H]2c2cccc([N+](=O)[O-])c2)n1', 'NS(=O)(=O)/C=C/COc1ccc2cccnc2c1', 'NS(=O)(=O)/C(C#N)=C\\c1ccc(-c2ccc(Cl)cc2)cc1', 'NS(=O)(=O)/N=C(\\[O-])c1n[nH]c(-c2ccoc2)n1', 'O=C(C[NH+]1CCN(C(=O)c2c(F)cccc2F)CC1)N1CCCC1', 'O=C(c1cn(Cc2ccco2)c2c1CCCO2)N1CCSCC1', 'O=C(c1cnc(Cc2ccccc2)s1)N1CCC2(CC1)Nc1ccccc1C2=C1CCOCC1', 'O=C(c1cn2ccccc2n1)N1CCN(Cc2ccccc2F)[C@H](C(=O)c2ccc(Cl)cc2)C1', 'O=C(c1cnn(-c2ccccc2)c1)N1CC2(CCCCC2)Oc2ccccc21', 'O=C(c1cn[nH]c1-c1ccccc1)N1CC(C(=O)NCCC23CC4CC(CC(C4)C2)C3)C1', 'N=CCCOc1ccc(O[C@H]2N=NC(=O)[C@@H]2C)cc1Cl', 'N=CC(=O)NC1CC[NH+](CCCn2ncc(C(F)F)n2)CC1', 'N=CCc1nnc(C(=O)N(C)Cc2ccccc2OCC(F)F)s1', 'N=CC1CCN(C(=O)Nc2c(Cl)cccc2Cl)C1', 'N=CCOC(=O)C1=c2ccccc2=[NH+][C@H]1Cn1ccnc1', 'N=CCNC(=O)c1cncc2c1CCO2', 'N=CC[C@@H]1CCCN1c1ncc(C(=O)[O-])cc1Cl', 'N=CCn1cccn1', 'N=CCS(=O)(=O)Cc1nc2nc(C)cc([O-])n2n1', 'N=CC[NH+]1CCCC[C@@H]1c1ccc(S(=O)(=O)C(F)F)cc1', 'Clc1nnnn1-c1ccccc1', 'Clc1nnns1', 'CC(=O)N1CCC(CCC(=O)NNC(=O)c2cn(C)nc2-c2ccccc2)CC1', 'CC(C)CS(=O)(=O)N(C)Cc1ccc(Br)o1', 'CCn1c(=S)sc2cc(Br)ccc21', 'CCSC1=NN(c2ncccc2[N+](=O)[O-])C(=O)[C@@H]1C', 'CCS(=O)(=O)c1ccc(NC(=O)c2cccc(C#N)c2)cc1', 'CCSc1ccc(/C=C/C(=O)[O-])cc1', 'CCS[C@@H]1CCCCN(Cc2ccccc2N2CCCC2)C1', 'O=[N+]([O-])c1ccc[nH+]c1NCCSc1ncccn1', 'O=[N+]([O-])c1ccc(OCc2ccc(C(=O)OCc3ccccc3)o2)cc1', 'O=[N+]([O-])c1cccc(OCCc2ccco2)c1', 'O=[N+]([O-])c1ccc2c(c1)[C@H]1C=CC[C@@H]1[C@@H](c1cccc(OCc3ccccn3)c1)N2', 'CNC(=O)c1cc(CN2CC[NH+](C)C[C@H]2c2cccc(Cl)c2)no1', 'ClCc1nnc(-c2ccco2)n1-c1cccc2ccccc12', 'ClCc1nnn(C2CCCCC2)n1', 'ClCc1nnsc1NCc1ccnn1C1CCOCC1', 'ClC[C@H]1C[NH+]2CCCC[C@H]1C2', 'ClC[C@H]1O[C@H]([N+]2CC2)C1(Cl)CCO', 'O=[N+]([O-])c1c(Br)cccc1[O-]', 'O[C@@H](c1cc(Cl)cc(Cl)c1Cl)c1ccccc1C1CC1', 'O[C@@H](Sc1cccs1)C(=O)NNS(=O)(=O)c1ccccc1Cl', 'O[C@@H](S[C@@H]1CCOC1)c1ccccc1', 'OCc1nc(C(=O)NCc2[nH]ccc2Cl)n[nH]1', 'OCc1nccn1CCCOc1cc(Cl)ccc1Cl', 'OCc1nc2ccc(F)cc2n1CCN1CCOCC1', 'OCc1ncn(-c2ccccc2)n1', 'OCc1ncsc1OC[C@H]1CN(Cc2ccccc2)C2(CCOCC2)C1', 'OCc1nc[nH]n1', 'COc1nc2scnc2cc1CNCc1cc(F)c(C)cc1OC', 'COc1nc2c(c(=O)[nH]1)[C@@H](c1c(F)cccc1Cl)[C@@H](C(=O)Nc1ccccc1Br)CS2', 'COc1nc2nc(C(=O)NCCCc3ccc(F)cc3)cn2cc1[N+](=O)[O-]', 'Cc1sc(=O)n(C)c1N1CCC(C(=O)NCCc2ccc(F)cc2)CC1', 'Cc1sc2ccccc2c1CC(=O)NC1CCC(CO)CC1', 'Cc1sc2sc(-c3ccc(S(=O)(=O)NC(C)C)cc3)nc2c1C', 'Cc1sc2ccccc2c1C(=O)N(C)[C@@H]1CCC[C@]2(CCOC2)C1', 'Cc1sc2nc(/C=C/c3cc(N)c(C#N)cn3)[nH]c(=O)c2c1C', 'O=[N+]([O-])c1c(Cl)cc(C(=O)c2ccc(OCc3ccc(Br)cc3)cc2)cc1F', 'O=[N+]([O-])c1c(-c2cccs2)nnn1-c1ccccc1', 'O=[N+]([O-])c1c(Cl)nc2ncon12', 'COC[C@H](C)OC(=O)CN1C(=O)c2c(C)cccc2N1c1cccc(Cl)c1', 'COC[C@H]1CCC[NH+](Cc2cccc(OC)c2OC)C1', 'Oc1cc(Br)ccc1Oc1ncc(Br)c(O)c1F', 'Oc1cc(F)c2nc([C@H]3CCCN3C(=O)Cc3ccco3)oc2c1', 'Oc1cc(F)nc(SCC(F)F)n1', 'O[C@@H](c1ccc(Cl)cc1Cl)c1ccccc1F', 'O[C@@H]([C@@H](O)c1ccccc1)c1ccc2c(c1)CCC2', 'O[C@@H]([C@@H]1CCOC2(CCOCC2)C1)[C@@H]1C[C@@H]1C', 'CN=C/C=C(\\C#N)C(=O)[C@@H](c1ccccc1)c1ccccc1', 'CN=N/NC(=O)[C@H](C)S(=O)(=O)C[C@@H](C)c1ccccc1', 'O=C(C1=C([O-])C(=O)N(Cc2c[nH]c3ccccc23)C1=O)N1[C@@H]2CC[C@@H]1Cc1ccccc1-2', 'O=C(C1CCC1)N1CCN(C(=O)C2CC2)CC1', 'O=C([C@H]1CC[NH+](CCC2CC2)C1)N1CCN(c2ccc(Cl)cc2)CC1', 'O=C([C@H]1[C@H](c2ccccc2O)C=C(CO)C1=O)NC1CC[NH+](Cc2ccco2)CC1', 'O=C([C@H]1c2ccccc2C[C@@H]1Oc1ccc2c(c1)OCCO2)N1CCN(Cc2cn3ccccc3[nH+]2)CC1', 'Clc1nnnn1Cc1cc(Br)ccc1O', 'Clc1nnnn1c1ccc(CSc2nnc3ccccn23)cc1', 'Clc1nnnn1-c1cccc(Cl)c1', 'Clc1nnnn1/N=C/c1ccc([N+](=O)[O-])cc1O', 'Clc1nnn(-c2ccc(F)cc2)c1I', 'Clc1nnn(C(C)(C)C2CCSCC2)c1N1CCCC1', 'Clc1nnn(Cc2ccc(F)cc2Cl)c1NOC(=O)CC1CCCC1', 'Clc1nnn(C2CCCC2)c1-c1ccc(Br)s1', 'Clc1nnn(C[C@@H]2CCCC[C@@H]2Cc2ccc(F)cc2)n1', 'Clc1nncn1Cc1ccc2c3c(cccc13)CCC2', 'Clc1nncn1c1cc(Cl)ccc1Cl', 'Clc1nncn1[C@@H]1CCOC2(CCCCC2)C1', 'Clc1nncn1-c1ccccc1', 'Clc1nncn1/c1nc2ccc(Br)cc2o1', 'C=C(Br)Cn1c(SCC(=O)N[C@H](C)c2ccccc2)nn(-c2ccccc2)c1=S', 'C=C(C)C[C@@H](NC(=O)CCc1ccc(F)cc1)C1CC1', 'C=C(Cl)C[NH+]1CCc2c(F)cc(F)cc2[C@H]1C', 'C=C([O-])c1ccc(N2CC(=O)NC3=C2CC[C@H]3C)cc1Cl', 'Oc1cc(O)ccc1OCCS(=O)(=O)N[C@@H]1CCc2ccccc21', 'Oc1cc(Br)c(/C=N/Nc2ccccc2)cc1F', 'Oc1cc(Br)ccc1OCc1ccccc1OC[C@@H]1CCCO1', 'Oc1cc(Br)c2ccccc2c1C(=O)N1CCCCCC1', 'Oc1cc(Br)cnc1OCc1ccnnc1Cl', 'Oc1cc(Br)cs1', 'O[C@@H](c1cscc1C(F)(F)F)C1CC1', 'O[C@@H](c1cccs1)[C@H]1COc2ccc(F)cc21', 'Clc1nnns1', 'O=[N+]([O-])c1ccn(Cc2noc(-c3ccc(F)cc3F)n2)c1', 'O=[N+]([O-])c1ccnc2nc(C3(c4ccc(Br)cc4Cl)CCC3)n(Cc3ccccc3)c(=O)c12', 'O=[N+]([O-])c1ccncc1[C@@H](OCC(=O)N1CCCC1)c1ccccc1', 'O=[N+]([O-])c1ccnc2ccccc12', 'O=[N+]([O-])c1ccn(-c2cc(C[NH3+])ncc2F)c1', 'O=[N+]([O-])c1ccn(Cc2cc(F)cc3c2OCOC3)c1', 'O=[N+]([O-])c1ccn([C@H](CO)c2ccc(Cl)cc2)c1', 'OC[C@@]1(COc2cccc(F)c2)CO[C@@H]2[C@@H]1[C@@H]1[C@H](C1)CN2CC1CC1', 'OC[C@@]1(c2cccs2)NC(=S)N1Cc1cccs1', 'OC[C@@]1(O)CCSC1', 'OC[C@@]1(NC2CC2)Cc2ccccc2C1', 'OC[C@@]1([C@H]2CCCN(C(=O)c3ccco3)C2)C(=O)C[C@@H]1Oc1ccccc1', 'OC[C@@]1([NH3+])CCC[C@H](Cc2ccncc2)C1', 'CC[S@](=O)[C@@H]1CCCC[C@@H]1NC(=O)c1cccnc1', 'ClC(F)(F)c1ccccc1OCc1nccn1Cc1ccccc1', 'ClC(=S)NNc1ccccc1[N+](=O)[O-]', 'ClC(F)/C=C/C(=O)N[C@@]1(CO)CCC[C@H]1CC1CC1', 'ClC(F)(F)C1CCN(Cc2csc(-c3ccc(F)cc3)n2)CC1', 'CC[C@@H](OC(=O)c1cccc([N+](=O)[O-])c1)[C@H]1CCCO1', 'CC[C@@H]1C[NH+](Cc2ccccc2)C[C@@H](C)C1', 'Clc1nnnn1-c1ccc(Br)cc1F', 'O=C(/C(CCCN1C(=O)NC2(CCCCCC2)N1c1ccccc1)Oc1ccccc1)c1ccccc1Cl', 'O=C(/C([O-])=N/S(=O)(=O)c1ccc(Br)cc1)Nc1ccccc1', 'O=C(/C(c1ccccc1Cl)N1CCN(c2ccccc2)CC1)c1cccc2c1ccc1ccccc21', 'O=C(/N=C(\\[O-])[C@H]1CC(c2cccc([N+](=O)[O-])c2[N+](=O)[O-])C1)OCc1ccc(Cl)cc1', 'O=C(/N=c1\\[nH]c2ncnc(-c3ccccc3)c2c2ccc(O)cc12)c1cncs1', 'O=C(CSC(c1ccccc1)c1ccccc1)N1CCc2ccccc21', 'O=C(CS(=O)(=O)c1ccc(Cl)c(Cl)c1)Nc1ccc(F)cc1', 'O=C(/N=c1\\sc2ccccc2c(=O)n1-c1ccccc1)C1CCN(C(=O)c2ccc(Cl)s2)CC1', 'O=C(COCC1CC[NH+](c2ccc(Cl)c(F)c2)CC1)/C=C/c1ccccc1', 'O=C(COc1ccccc1C(=O)[O-])NCc1ccccc1', 'O=C(/C=C/c1ccc2c(c1)OCO2)NCc1cccs1', 'O=C(/N=C/c1csc2ccccc12)N[C@@H](CC#N)c1cccc(Cl)c1', 'O=C(/N=C\\c1ccccc1[N+](=O)[O-])Nc1ccc([N+](=O)[O-])cc1', 'O=C(Cc1ccccc1)NC[C@@H](c1cccs1)[C@H]1CCCO1', 'O=[N+]([O-])c1c(OCCO[C@@H]2CCO[C@@H]2c2ccccc2)c2ccccc2o1', 'O=[N+]([O-])c1c(O)cc(NC[C@@H]2CCC[C@@H]2O)nc2n1CCCCC2', 'O=[N+]([O-])c1c(O[C@@H]2CCCCCC2=O)cc(N2CCOCC2)c2ccccc12', 'Oc1ccccc1N1C(=O)c2ccccc2C1=O', 'Oc1ccccc1Cc1ccccc1OCc1ccccc1Cl', 'Oc1ccccc1OC1CC(C(F)(F)F)O[C@@H]1c1ccccc1', 'Oc1ccccc1[C@@H]1N[C@@H](c2cc(OCc3ccccc3)ncn2)CCO1', 'Oc1ccccc1-c1cccc2cccnc12', 'Oc1ccccc1[C@H]1CN(c2ccccc2)CCN1CCn1cc[nH+]c1', 'Oc1ccccc1/C=N/NC(=O)c1ccc([N+](=O)[O-])cc1', 'Oc1cccc2ccccc12', 'Oc1cccc2ncnc(C(=O)Nc3cccc(C[S@](=O)c4ccccc4F)c3)c12', 'Oc1cccc2nnc(-c3ccccc3F)n12', 'Oc1ccc(F)c(-c2nc(C(=O)N3CCCN(C4CC4)CC3)cnc2-n2cccc2)c1', 'O[C@@H](CCc1ccccc1)C(=O)Nc1ccccc1C(=O)[O-]', 'O[C@@H](C(=O)NC[C@@H]1CCC[C@H](S(=O)(=O)N2CCCC2)C1)[C@H]1CCS(=O)(=O)C1', 'O[C@@H](Cc1cccc(Br)c1)[C@@H]1CCCO1', 'O[C@@H](COCc1cc(-n2cccc2)c[nH]1)c1ccc(F)cc1', 'O[C@@H](CNc1cnccn1)c1ccc(Cl)cc1', 'O[C@@H](C[C@@H]1CCOC2(CCCCCC2)C1)C(F)(F)F', 'O[C@@H](Cn1cccn1)[C@H]1CSCCS1', 'O[C@@H](C#Cc1cscn1)C1CC1', 'O[C@@H](CSc1n[nH]c2c1CCCC2)c1ccccc1O', 'O[C@@H](C[C@H]1CCCN(S(=O)(=O)c2ccc(C(F)(F)F)cc2)C1)c1cccnc1', 'O[C@@H](C[NH+]1CCC[C@H]1c1ccccc1)C1CC1', 'CNc1[nH+]ccc(N)c1S(=O)(=O)NNc1cccc2ccccc12', 'Cc1ncsc1C(=O)N1CCC[C@@H](Oc2nc3ccccc3cc2CO)C1', 'Cc1nccn1-c1ccc(NC(=O)N[C@H](C)c2c(C)nn(C)c2C)cc1O', 'COc1nn(-c2ccccc2)c(C)c1C(=O)NC[C@@H](c1cccs1)[NH+](C)C', 'COc1nn(C)cc1CN[C@@H](C)c1ncc(-c2ccccc2)n1C', 'CN[C@H](CO)C1(C)CC1', 'CN[C@H](NC(=O)NC[C@H](O)[C@@H]1CCCO1)c1ccccc1', 'CN[C@H](c1cccc(OC)c1)c1cc(F)cc(F)c1', 'ClCc1csc(C2CC2)n1', 'ClCc1csc2c1COCCCO2', 'N#C[C@H](C(=O)[C@@H]1CCCN(C(=O)Cc2ccccc2Cl)C1)c1ccccc1', 'N#C[C@H]1C(=O)NC[C@@H](C(=O)c2ccccc2)N1CNc1cccc(Cl)c1', 'Cc1nnsc1CN1CCN(C(=O)c2ccc(Cl)cc2)CC1', 'Cc1nnc(SCCN2C(=O)N[C@]3(CC[C@@H](O)CCC3)C2=O)s1', 'NS(=O)(=O)Cc1csc(NC(=O)c2cccc([N+](=O)[O-])c2)n1', 'NS(=O)(=O)/C(=C\\Cc1cccc(C(F)(F)F)c1)c1cccc2ccccc12', 'NS(=O)(=O)/C(C#N)=C\\c1cccc(Cl)c1', 'Nc1cnn(CCC(=O)N2CC[C@@H](C(=O)[O-])[C@H]2C)c1[N+](=O)[O-]', 'Nc1cn[nH]c1-c1ccc(NC(=O)c2ccccc2O)cc1', 'NS(=O)(=O)C[C@@H](SCC(=O)c1ccsc1)C(=O)[O-]', 'NS(=O)(=O)C[C@@H]1[NH2+]CC[NH+](Cc2ccccc2)C1', 'COc1ncnc(NCc2ccccc2OCC2CC2)c1C', 'COc1ncc(C)cc1OC(=O)[C@@H]1Cc2ccccc2CN1C(=O)C(C)C', 'COc1ncccc1COC(=O)c1cccc2cc[nH]c12', 'COc1ncc2nc(NC(=O)Nc3cc(C(C)(C)C)cs3)nn2c1', 'CCNC(=O)N[C@H](C)c1nc2ccccc2n1CCN1CC[NH+](C)CC1', 'CCN(C[C@@H]1CCOC1)C(=O)NCc1cccc(C[NH+]2CCCC2)n1', 'CCNc1cccc(NC(=O)NC2CN(C(=O)CCc3ccc(C)cc3)C2)c1', 'CCN1C(=O)N=C(NNc2ccc(C)cc2C)C1=O', 'CCNS(=O)(=O)C[C@@H](C)C#N', 'CCN[C@](CCC(F)(F)F)C(=O)OC', 'Oc1cc(-n2ncc(-c3ncnn3CCOc3ccccc3)n2)cc(S[C@H]2CCOCC2)c1', 'Oc1cc(-n2nnc3c(F)cccc32)c([N+](=O)[O-])c2ccccc21', 'Oc1cc(OCC(F)F)ccc1Oc1cccc(Cl)c1', 'Oc1cc(OC(=O)c2ccc(Cl)cc2F)ccc1N[C@H](c1ccccn1)C1CCCC1', 'Oc1cc(OC2CC[NH+](Cc3ccccc3F)CC2)cc(SCC(=O)NC2CCC2)c1', 'Oc1cc(OCOc2cc(-c3ccccc3)n[nH]c2=O)cs1', 'Oc1cc(OC[C@@H]2CC[C@@H](C(F)(F)F)N2)c2ccccc2c1C1CCCCCC1', 'Nc1[nH]ncc1C[NH+]1CCC(Oc2ccccc2F)CC1', 'O=C(CCCC[NH3+])C1CCCCC1', 'O=C(CC(=O)Nc1ccc(Cl)cc1F)Nc1cccc(N2CCCC2)c1', 'O=C(CCc1ccccc1)NCCc1nnc(-c2ccc(Cl)s2)n1-c1cccnc1', 'O=C(CC1CC1)N1CCC(CC(=O)N(CCO)C2CC2)CC1', 'O=C(CCOc1ccccc1Cl)[C@@H]1N=NN=C1c1ccc2c(c1)OCO2', 'O=C(CCNC(=O)Cn1c(N(C)Cc2ccccc2)nn(-c2ccccc2)c1=O)N1CCOCC1', 'O=C(CC[C@@H]1CCCO1)NCCc1cccc([N+]2=CCCC2)c1', 'O=C(CCS(=O)(=O)c1cccc(C(F)(F)F)c1)NC[C@H](O)c1ccccc1', 'O=C(CC[NH+]1CCC[C@@H]1c1ccco1)Nc1ccccc1[N+](=O)[O-]', 'OC[C@H](CO)Nc1ccc(F)c(Cl)c1', 'OC[C@H](c1ccccc1)[NH+]1CCN(c2ccccc2)CC1', 'OC[C@H]([S@@](=O)C1CCOCC1)N1CCCc2ccccc21', 'OC[C@H](O)c1ccc(C(F)F)cc1Br', 'OC[C@H](Nc1cccc(F)c1)C(=O)N1CCCC1', 'OC[C@H](SC1=NCCS1)c1ccccc1', 'OC[C@H](Cl)c1ccc2ccccc2c1', 'OC[C@H](Br)CC12CC3CC(CC(C3)C1)C2', 'OC[C@H]([NH2+]Cc1ccccc1)c1cc(Cl)sc1Cl', 'N#C[C@@H](C(=O)c1ccc(CN2CCSCC2)cc1)c1ccoc1', 'N#C[C@@H]1C(=O)N(CCC(=O)Nc2cc(F)ccc2F)C(=O)N(C(C)(C)C)C1=O', 'ClC(c1ccccc1)C1(C(=O)NC2CCCC2)CC(CC2CCCC2)C1', 'ClC(=S)c1ccc(N2CCC(COc3ccccc3)CC2)cc1', 'ClC(/C=N/O)c1c[nH+]ccc1N1CCOCC1', 'ClC(/N=C1/CSC[C@@H]1N1CCCC1)c1ccc(SC(F)F)cc1', 'Nc1nnnn1CC[NH+]1CCN(Cc2ccccc2)CC1', 'Nc1nnc(Cc2cccs2)n1-c1ccccc1', 'N[C@@H](CCSCc1ccco1)C(=O)Nc1ccccc1', 'N[C@@H](C(=O)Nc1ccccc1C(F)(F)F)[C@H]1CCCO1', 'N[C@@H](Cc1ccc2c(c1)C(=O)CCC2)c1cc(OC(F)(F)F)ccc1Cl', 'N[C@@H](C1CCC(c2cccs2)CC1)N1CCN(c2ccccc2F)CC1', 'N[C@@H](COCc1ccc(F)cc1)c1cc(Cl)ccc1Cl', 'N[C@@H](CNC(=O)c1cccn2ccnc12)c1cccnc1', 'N[C@@H](C#N)c1ccccc1Cl', 'N[C@@H](C[NH+](C)CC(=O)N1CCOCC1)c1ccc(F)cc1', 'N[C@@H](C[NH3+])C(=O)NC[C@@H]1CCCO1', 'N#C[C@@]1(C2CC2)Nc2cccnc2N1', 'COC(=O)N1CC[C@@H]1CC(=O)Nc1ccccc1Oc1ccccc1F', 'COC(CNC(=O)N1CC[C@H](C)[C@@H](O)C1)c1cccc(F)c1', 'COc1nn[nH]c1CCC(=O)N1CCC[C@@H](C(=O)NCc2ccccc2F)C1', 'COc1nn[nH]c1C(=O)NC[C@@H](C)c1cccc(C)c1', 'Cc1snnc1CNC(=O)N(C[C@H]1COCCO1)C1CC1', 'Clc1cc(Br)c([N+](=O)[O-])cc1Sc1ncccn1', 'Clc1cc(/C=N/N=C\\c2ncn(-c3ccccc3)n2)c[nH]1', 'Clc1cc(/N=N/S(=O)(=O)c2ccc(C(F)(F)F)cc2)cc(Cl)c1Cl', 'ClCc1n[nH]c(-c2ccccc2)c1N1CCc2cc(F)ccc2C1', 'ClCc1n[nH]c(O)c1C(F)(F)F', 'ClCc1n[nH]cc1C[NH+]1CCN(c2ccc([N+](=O)[O-])cc2)CC1', 'ClCc1n[nH]c2c1CCC2', 'C[C@@H](NC(=O)Nc1ccccc1)[C@@H]1CCCN(c2ncccn2)C1', 'C[C@@H](NS(=O)(=O)c1ccc(F)c(F)c1)c1nc(-c2ccccc2)no1', 'C[C@@H](Nc1cccc(Cl)c1)[C@@H]1CCC[NH2+]C1', 'C[C@@H](N[C@@H](CC#N)c1ccccc1)c1ccccc1Br', 'Oc1cccc2cc(C(=O)N3CCCCC3)c3nc(-c4ccccc4)c(=O)[nH]c3c12', 'Oc1cccc2cc(C[NH2+][C@@H]3CC[NH+]c4ccc(Cl)cc43)oc12', 'Oc1cccc2cc(OCC(=O)Nc3cc(C(F)(F)F)ccc3O)ccc12', 'Oc1cccc2cc([C@@H](CNS(=O)(=O)c3ccc4c(c3)OCO4)C(=O)[O-])c(=O)n(Cc3ccccc3)c12', 'Oc1cccc2cc(-c3ccccc3)cnc12', 'Oc1cccc2cc(/C=C/C(=O)c3ccc(F)c(F)c3)cnc12', 'COCc1cccc(NC(=O)C(=O)N2CCC[C@H](C(=O)Nc3ccccc3)C2)c1', 'COCCC(=O)N1CCc2ccc(-c3ncoc3C)cc21', 'COCC(=O)NC1(N(C)Cc2ccc(F)c(F)c2)CCC1', 'COCCOc1ccc(CN2CCC[NH2+]C3(CCCCC3)C2)cc1', 'COCC[C@@H](C)C(=O)N[C@@H]1CC[NH+](Cc2ccccc2)C1', 'COCCn1ccc(C[NH2+][C@@H]2CCCc3c(C)cc(C)cc32)c1', 'COCCS[C@H](C)C(=O)N[C@H]1CCc2ccc(C)cc21', 'COCC[NH+](C)CCn1c(-c2ccc(F)cc2)nc2ccccc21', 'COCC[C@@]1(Cc2ccc(F)cc2)NCCn2cccc21', 'O=C([C@H](SCc1ccccc1O)Br)c1ccccc1', 'O=C([C@H](Sc1nccc(Cl)c1F)N1CCCC1)c1ccc2c(c1)OCCO2', 'O=C(/C=C/c1cnccn1)Nc1cccc(Oc2ccccc2)c1', 'O=C(/C=C\\c1cccs1)NC1CCN(c2ncccn2)CC1', 'O=C(/C=C1\\C(=O)Nc2ccccc21)c1ccccc1Cn1cccn1', 'O=C([O-])[C@@H](Cc1ccccc1)c1cc(Cl)ccc1F', 'O=C([O-])[C@@H]1CC(=O)N(CCSc2ccccc2)C1', 'O=C(c1c[nH]c2ccc(Br)cc12)N1CCSCC1', 'O=C(c1c[nH]nc1-c1ccccc1)N1CC[C@@H](c2ccccc2)[C@H]1Cc1cccs1', 'O=C(/C(c1cccc(F)c1)NCc1ccccc1)[C@H]1CC=CCC1', 'O=C([C@H]1c2ccccc2S(=O)(=O)C[C@@H]1NC(=O)Cc1c[nH]c2ccccc12)N1CCCCC1', 'Oc1ccc([N+](=O)[O-])cc1NCc1cccnc1', 'Oc1ccc(/C=N/Nc2ncnc3ccccc23)cc1', 'Oc1ccc([N+](=O)[O-])cc1Sc1ccccc1C(=O)Nc1nc(Cn2ccc(C(F)(F)F)c2)n[nH]1', 'Oc1cc(-n2cccc2)ncc1S[C@@H]1CC[NH2+]C1', 'Oc1cc(-n2cncn2)ccc1NN1CCOCC1', 'O=[N+]([O-])c1ccc(CNC[C@@H]2CCC[NH2+]2)cc1', 'O=[N+]([O-])c1ccc(OCC(=O)N2CCCC[C@H]2c2ccccc2)cc1', 'O=[N+]([O-])c1ccc(N[C@H](c2ccccc2)C2CC2)cc1F', 'O=[N+]([O-])c1ccc(F)cc1NC(=O)COC(=O)c1ccc2c(c1)OCO2', 'O=[N+]([O-])c1ccc(-c2noc(-c3ccc([N+](=O)[O-])cc3)n2)cc1Cl', 'O=[N+]([O-])c1ccc(S(=O)(=O)NC[C@@H]2CCN(c3ccccc3)C2)cc1', 'O=[N+]([O-])c1ccc([C@H]2C[C@H]2CN2CCOCC2)cc1', 'O=[N+]([O-])c1ccc(/C=C/CCCSc2ccccn2)c([N+](=O)[O-])c1', 'O=C(c1c[nH]c(=O)[nH]c1=O)N[C@@H]1CCS(=O)(=O)C1', 'O=C(c1c[nH]cn1)N1CC[C@@](CO)C1', 'O=C(c1c[nH]c2ccccc12)N(C[C@@H](O)c1ccc(Br)cc1)C1CCC1', 'Oc1ccccc1CCN1CCC([NH2+]Cc2ccc(CO)cc2)CC1', 'Oc1ccccc1C(=O)OCc1ccc(F)cc1', 'Oc1ccccc1Cc1cc(F)ccc1Br', 'Oc1ccccc1C1=NN(c2ccccc2)[C@H](c2ccccc2)C1', 'Oc1ccccc1COc1ccccc1Oc1ccc(F)cc1F', 'Oc1ccccc1CN1CC[NH+]([C@H]2CCOc3ccccc32)CC1', 'Oc1ccccc1Cn1ccnc1Sc1ncnc([O-])c1', 'Oc1ccccc1C[C@H]1CCC[NH+](C2C[NH+](c3nc(OCc4ccccc4)cs3)C2)C1', 'Oc1ccccc1C[NH+]1CCC[C@@H]1CN1CCCC1=O', 'Nc1cn[nH]c1I', 'O=C([O-])/C=C/c1cccc(O[C@H]2CCN(c3ccccc3)C2=O)c1', 'O=C(/N=c1\\sc(C(Cl)(Cl)F)nn(CN2CCCC2)c1=O)c1cc(Br)sc1Cl', 'O=C(/N=c1/[nH]ccn1C1CC1)Nc1ccccc1N1CCN(c2ccccc2F)CC1', 'O=C(/N=c1cc2oc3ccccc3c(=O)n1[C@@H]2c1cccs1)c1ccc(Cl)cc1', 'Clc1cccc(C[C@@H](Cc2cccnc2)[NH+]2CCN(C(=O)c3ccccc3)CC2)c1', 'Clc1ccccc1CSc1nccc(C(F)(F)F)n1', 'Clc1cccc2c1NC[C@H](c1ccccc1)CN2Cc1ccccc1', 'O=[N+]([O-])c1cc(OC/C(NC[C@H]2CCCO2)c2ccccc2)ccc1Cl', 'O=[N+]([O-])c1cc(O)cc(C[NH+]2CC[C@H]3SCCC[C@@H]3N2)c1', 'O=[N+]([O-])c1cc(O[C@@H]2CCCC[C@@H]2CCC[NH+]2CCCC2)c(NC2CC2)cc1', 'NS(=O)(=O)/N=C(\\NCc1ccc(F)cc1)C1CCCCC1', 'NS(=O)(=O)/N=c1\\sc(N)c(-c2ccc(Br)cc2)c(=O)c2ccccc12', 'Oc1cc(F)c(-c2ccco2)cc1N1CCC[C@@H]1C[NH+]1CCCC1', 'Oc1cc(F)ccc1C(F)(F)C(F)(F)F', 'ClC[C@@H]1OCC[C@@H]1c1ccc(F)cc1', 'O=C([C@H](CC1CC1)N(Cc1ccc(Cl)cc1)C1CC1)NCC(=O)Nc1ccccc1', 'O=C([C@H](C(=O)[O-])c1ccc(Cl)cc1)c1ccc(F)cc1', 'O=C([C@H](Cc1ccccc1)N1CCOCC1)[C@H]1CN2CCCC[C@@H]2CO1', 'O=C([C@H](CNC(=O)CSCc1ccc(Cl)cc1)N1CCCC1)Nc1ccccc1Cl', 'O=C([C@H](C#N)c1ccc(Cl)cc1)N1CCN(c2ccc(Cl)cn2)CC1', 'O=C([C@H](CSc1ccccc1)c1ccccc1)N1CCCC1', 'O=C([O-])c1ccc(CN[C@H]2CC(=O)N(c3ccccc3)C2)c(F)c1', 'O=C([O-])/C=C/C=C/C=C/C1=CCCCC1', 'O=C([O-])/C(=N\\OCc1cccs1)c1ccc(Cl)cc1', 'Oc1ccc(/N=C/c2cccc(Cl)c2C)cc1Cl', 'Oc1ccc([N+](=O)[O-])c(Br)n1', 'Oc1cc(-n2nnc(-c3ccccc3)n2)c2ccccc2c1OCC[C@@H]1CCCC[NH+]1CC1CC1', 'Oc1cc(-n2nnnc2SCc2ccccc2)ccc1OCc1cccnc1', 'Oc1cc(-n2nc(C(F)(F)F)cc2Cn2ccccc2=O)ccc1O', 'Oc1cc(-n2nc3c(=S)[nH]c(-n4cncn4)c3c2=O)ccc1Cc1ccccc1F', 'Oc1cc(-n2ncc(Br)c2Cl)ccc1-c1ccccc1O', 'Oc1ccc(/C=C/C[NH2+]Cc2ccccc2O)cc1', 'Oc1ccc(/C=N/N=C2\\C(=O)N(c3ccccc3)C(=O)NC2=S)cc1', 'Oc1cc(F)nc(N2CCCCCCC2)c1', 'Nc1[nH]nc(-c2cccc(Cl)c2)c1N', 'Nc1[nH]nc(SCC(=O)Nc2nc3ccccc3[nH]2)c1Cl', 'Nc1[nH]nc(C2CC[NH+](Cc3ccccc3C(F)(F)F)CC2)n1', 'Nc1[nH]nc(N2CC[NH+](Cc3cnn4ccnnc34)CC2)c1-c1ccc(Cl)cc1', 'OC1(C[NH2+][C@@H]2CCOC3(CCCCC3)C2)CCCCC1', 'OC1(NC(=O)CN2CSc3ccccc32)CCCC1', 'OC1(SCCC(=O)Nc2ccc(Br)cc2Cl)CCCCC1', 'OC1([C@H]([O-])c2ccc(-n3cccn3)cc2)CCOCC1', 'C=C1c2ccc(OCC(=O)OCC(=O)N/N=C/c3ccccc3)cc2O1', 'C=C1c2ccccc2C(=O)O[C@H]1CC(=O)N1CCC2(C1)OCCO2', 'Cc1sn(-c2ccccc2)c1COC(=O)c1cnn(C)c1C', 'Cc1snc(C[NH+]2CCC(C(=O)NCc3scnc3C)CC2)n1', 'Cc1sncc1CNc1ccnc(OCC2CC2)c1', 'O=C(c1cc([N+](=O)[O-])ccc1N1CCN(c2ccccc2)CC1)N1CCC(OC2CCOCC2)CC1', 'O=C(c1cc([S@](=O)Cc2ccccc2F)c[nH]1)Nc1ccc2nc(C3CC3)oc2c1', 'O=C(c1cc(F)ccc1F)N(CCCN1CCOCC1)S(=O)(=O)c1cccc(Cl)c1', 'O=C(c1cc(Cl)ccn1)N1CC[NH2+]CC1', 'O=C(c1cc(Br)ccc1O)N1CC[C@@](O)(C(F)(F)F)C1', 'O=[N+]([O-])c1ccnc2c1C[NH+](Cc1ccc(C(F)(F)F)c(Cl)c1)CC2', 'N#Cc1cccc(NC(=O)COc2ccc(Br)cc2)c1', 'N#Cc1nccn1/c1ccc(OCc2ccccc2)cc1', 'N#C[C@@]1(O)CCc2c(s1)C[NH+](Cc1csc(-c3cccs3)c1O)CC2', 'N#C[C@@]12CCC(CCNC(=O)c3sc4ncc(Cl)cc4c3[C@H]1C(=O)c1ccccc1)C2', 'C=C[C@@H](C)C(=O)NCCNC(=O)c1n[nH]c(C)c1C', 'C=C[C@@H](O)CN1CCc2ccc(C(=O)NC)cc2C1', 'C=C[C@@H](NC(=O)c1ccc(F)cc1)c1ccccc1OC', 'C=C[C@@H](Sc1cccc(F)c1)C(=O)N/N=C/c1ccc(C)cc1', 'C=C[C@@H]([NH3+])c1ccc(OC[C@@H]2CCCO2)cc1', 'Oc1ccc([C@H](CN2CCCCCC2)c2ccccc2)c(F)c1', 'Oc1ccc([C@H]2NC(=O)c3ccccc32)c2ccccc12', 'C[C@@H](c1ccccc1)S(=O)(=O)CC(=O)NC(=O)N[C@@H](C)Cc1ccccc1', 'C[C@@H](c1nnc(S[C@@H](C)C(=O)N2CCCc3ccccc32)s1)n1cccn1', 'Clc1nnn(-c2ccccc2)n1', 'Clc1nnnc(-c2ccco2)n1', 'Clc1nnnc(-n2nc(-c3ccc(F)cc3)cc2Br)n1', 'Clc1nnn(-c2ccccc2)c1N[C@H]1CCOc2ccccc21', 'Clc1nnn(-c2nccc(Cl)c2Cl)c1NS(=O)(=O)c1cccc(F)c1', 'Clc1nnc(CCC[C@H]2CCCO2)nc1C1CC1', 'Clc1nnc(C2CCCC2)n1N1CCCC1', 'Clc1nnc(COc2ccccc2)n1Cc1ccc2c(c1)OCCO2', 'Clc1nnc(CNc2nccc(OCc3cccs3)n2)o1', 'Clc1nnc(CSc2c[nH]c3ccccc23)[nH]1', 'Clc1nnc(C[NH+](C2CC2)[C@H]2CCS(=O)(=O)C2)s1', 'Clc1nnns1\n', 'CCO[P@](=S)(C(=O)[O-])c1ccccc1', 'CCOc1ccc(N2C[C@H](C(=O)NC[C@@H](Br)CCO)CC2)cc1', 'O=C(/C(Cc1cccs1)Nc1cccc2ccccc12)c1cccnc1', 'O=C(/C(CO)(CC1CC1)c1ccc(Br)cc1)c1ccccc1', 'O=C(/C(CNN=C1CCCC1)c1ccc(Cl)cc1)CC1CC[NH+](C2CCCCC2)CC1', 'CNS(=O)(=O)c1cccc(C(=O)Nc2nc(C3CC3)cc(C)c2[N+](=O)[O-])c1', 'CNS(=O)(=O)c1ccc(C(=O)[C@@H](C#N)C(=O)CC(C)(C)CC)cc1', 'CNc1[nH+]cccc1C(=O)N1CC[C@@H](c2ccccc2)C1', 'CNc1cccc(CNC(=O)Nc2ccc(F)c(Cl)c2)c1O', 'CNc1ncnc(S[C@@H](C)C(=O)NC2CC2)c1C1CC1', 'CN(CC(=O)N[C@H]1CCN(c2ccccc2)C1=O)C(C)C', 'CN(C(=O)CSc1nnnn1C)c1cc(OC)ccc1OCC=C1CCCCC1', 'CN(C)C(=O)c1ccc(NC(=O)c2ccc([N+](=O)[O-])o2)cc1', 'CN(Cc1ccc2ccccc2c1)C(=O)CN1C(=O)N[C@](CNS(=O)(=O)c2ccccc2)C1', 'CN(C[C@H](C[NH3+])c1ccc([N+](=O)[O-])cc1)C(C)(C)C(=O)NC1CC1', 'Cc1ncc([C@H](C)[NH2+]C(C)(C)C)s1', 'Cc1ncc2cc(C(=O)N[C@H](C#N)C(N)=O)cc(=O)cc2n1', 'Cc1ncccc1C(=O)Nc1cncc(C(F)(F)F)c1', 'Cc1nccn1-c1cccc(C[C@H]2CCC[NH2+][C@H]2C)c1', 'Clc1cc(Cl)ccc1-n1cccc1C1CC[NH+](Cc2ccccc2)CC1', 'Clc1cc(NS(=O)(=O)c2ccccc2)c2ccccc2n1', 'Clc1cc(NCCc2ccccc2Cl)ncn1', 'Clc1cc(Nc2ccc(Cl)c(Cl)c2)c2ccccc2c1Cl', 'Clc1cc(N2CCN(S(=O)(=O)c3ccc(Cl)cc3Cl)CC2)ccc1[O-]', 'Clc1cc(Br)ccc1Sc1ccc(C=O)cc1', 'Clc1cc(Cl)c(CN2CCOCC2)cc1O', 'Clc1cc(Cl)nc(N2CCC(CSc3ccc(O)cc3)CC2)c1', 'Clc1cc(F)cc(N2CCCC[C@@H]2Cc2ccccc2)c1', 'Clc1cc(CC(=O)N2CCN(CCn3cncn3)CC2)ccc1N1CCCC1', 'Clc1cc(C(F)(F)F)nc(-c2cccc(F)c2Cl)n1', 'Clc1cc(C2=NN=C([S-])N2)ccc1OCO', 'Clc1cc(CO)ccc1C(F)(F)F', 'Clc1cc(CNC[C@H]2C[NH2+]CCO2)ncn1', 'Clc1cc(C[C@@H]2Cc3ccc(Cl)cc3O2)n[nH]1', 'Clc1cc(CSC[C@@H](CSc2ccccc2)c2ccc(Br)cc2)c(F)c2c1CCCC2', 'Clc1cc(CCl)ccc1Br', 'Clc1cc(C[NH+]2CCC[C@@H]2c2ccc[nH]2)ccc1F', 'Clc1cc(C[NH2+]C[C@@H]2CCCCO2)c[nH]1', 'Clc1nnnc(NC[C@@H]2CCCO2)n1', 'Clc1nnnc(NNc2ccc(S(=O)(=O)N3CCCCC3)cc2)c1Cl', 'Clc1nnnc(N2CCOCC2)c1NCC1OCOC1', 'Clc1ccc([C@@H](NC[C@@H]2CCCCO2)c2ccc(Cl)cc2)cc1', 'Clc1ccc(-c2nnc(-c3ccc(=O)ccc3CO)s2)c(O)c1', 'ClCc1ccc(C2=CCN(Cc3ccccc3)CC1)CCCC2', 'ClCc1ccccc1OCc1cccc2ncccc12', 'ClCc1ccc2c(c1)OCC[C@@H]2c1ccc(CN2CCCCC2=O)cc1', 'ClCc1ccc[nH]1', 'OCc1nnsc1NC[C@H](O)c1ccccc1F', 'OCc1nn(-c2ccc(F)cc2)c2c1C[C@@H](NCc1ccco1)C2', 'OCc1nncc(-c2ccco2)c1C1CC[NH+](Cc2ccccc2)CC1', 'OCc1nnn2c1N[C@]1(C=CCC1)[C@H]2CC(=O)NC1CCCCCC1', 'Clc1nncc(S(=O)(=O)N[C@@H](Cc2ccccc2)c2ccccc2)c1', 'Clc1nncc(Sc2nnc3ccc(F)cc3c2Cl)n1', 'Clc1nncc(S[C@@H]2CC[C@@H](COc3cc([N+](=O)[O-])ccc3F)C2)n1', 'Clc1nncc(S/c2ccc(F)cc2F)n1', 'NS(=O)(=O)Nc1cc([N-]S(=O)(=O)c2cc(F)ccc2F)ccc1F', 'CCc1[nH]c(CCl)c(C#N)c1N', 'CCc1ccc2nc(N(Cc3ccco3)C(=O)c3ccc(O)c(F)c3)[nH]c2c1', 'CCc1nc2n(n1)C=CC(C(=O)Nc1nc(C(C)(C)C)cs1)C(=O)C(C)(C)C2', 'Oc1cccc2nn[nH]c12', 'Oc1cccc2nncn12', 'Oc1ccc(F)c([O-])c1-c1ccc([N+](=O)[O-])cc1Cl', 'Oc1ccc(F)cc1CNc1cc(Br)ccc1F', 'Oc1ccc([N+](=O)C2C[C@@H](C3CCN(c4ccccc4F)[C@@H]3c3ccccc3)c3ccccc32)cc1', 'Oc1ccc([N+](=O)[O-])c(C(F)(F)F)c1', 'Oc1ccc([N+](=O)c2ccccc2)c(F)c1Oc1ccccc1F', 'Oc1ccc(/N=c2[nH]c3ccccc3n2Cc2ccccc2)cc1OCC1CCN(CO)CC1', 'Oc1ccc(/N=c2\\sc(Cl)cc(Cl)c2C[NH+](CC(=O)[O-])c2ccccc2)cc1', 'Oc1cc(O[C@@H](C[NH+]2CCCCC2)c2ccccc2)ccc1OC[C@@H]1CCCO1', 'Oc1cc(O[C@@H]2CCO[C@@]3(CCOC3)C2)c(Cl)cc1Cl', 'NS(=O)(=O)/C=C/c1ccc2c(c1)OCO2', 'NS(=O)(=O)/C=c1/sc2nc[nH]c2cc1NC(=O)c1ccccc1F', 'ClC(c1cccs1)c1cc(CC2CCN(C(=O)c3ccc(F)cc3)CC2)nc2ccccc12', 'ClC(=S)[N+]1(c2cc(Br)ccc2F)CC1', 'ClC(=S)c1ccc(N[C@@H]2CCc3c(Cl)cccc32)cc1', 'ClC(=S)O/C(=C/c1ccc(N2CCCC2)cc1)c1ccc(F)cc1', 'ClC(=S)Nc1ccc2c(c1)OCCO2', 'ClC(=S)[C@@H](NC(=S)Nc1cc(F)ccc1F)c1ccc(Cl)c(Cl)c1', 'ClC(=S)Sc1ccc(N[C@H]2CCCC[C@H](C(=O)[O-])C2)cc1', 'N[C@@H](S/C=C/CO)c1ccc(Cl)cc1', 'N[C@@H](Sc1nc(-c2ccccc2F)n[nH]1)C1CC1', 'O=[N+]([O-])c1c(F)c(Cl)cnc1NCc1ccc(N2CCOCC2)cc1', 'Clc1nnn(-c2ccc(F)cc2)c1N1CCOCC1', 'Clc1nnn(-c2cncc(C[NH+]3CCOCC3)c2)c1Cl', 'Nc1noc(C[NH+]2CCC[C@H](c3nnc(Nc4cccnc4)n3N)CN2)c1Cl', 'Nc1nonc1C(=O)N1CCC(c2ncc(C(C)(C)C)nc2)CC1', 'Oc1cccc(-c2cc(CCOc3cccc4ccccc34)no2)c1', 'Oc1cccc(-n2ccc3ncnn32)c1', 'O=C(CNC(=O)C1CCCC1)N1N=C(c2ccco2)C(=O)[C@@H]1C(c1ccc(F)cc1)c1cccs1', 'O=C(CN(CCc1ccccc1)C1CC1)Nc1ccccc1Cl', 'O=C(CNc1cccc(OC(F)(F)F)c1)N1CCN(S(=O)(=O)c2ccccc2)CC1', 'O=C(CNS(=O)(=O)c1ccc(F)cc1)NC1CCCC1', 'O=C([O-])c1[nH]nc(-c2ccc([N+](=O)[O-])cc2)c1Br', 'O=C([O-])c1cc(C2CC2)on1', 'O=C([O-])c1nc([C@@H]2CCCN(c3ccccc3)C2)nc2c(C)cccc12', 'Oc1cccc2ncoc12', 'Oc1cccc2nc(CN3CCCC[C@H]3C(=O)[O-])nn12', 'Oc1cccc2ncccc12', 'Oc1cccc2ncnc(Nc3cccc(I)c3)c12', 'O=C(/C=C/O)C(=O)c1ccc(Cl)c(F)c1', 'O=C(/C=C/c1ccc(Br)cc1)N(C[C@H]1CCCO1)c1ccccc1', 'O=C(C[NH+]1CCN(C(=O)CCC2C[C@@H]3CCCC[C@H]3C2)CC1)NC1CCCCC1', 'Oc1ccc(F)cc1CN[C@@H](C1CC[NH2+]CC1)c1ccc(F)cc1', 'ClCc1nc([C@@H]2CCC[NH2+]C2)cs1', 'ClCc1ncoc1Cl', 'ClCc1ncnn1-c1ccccc1', 'C[C@@H]([NH2+]CC(=O)NC1(Cc2cccs2)CCC1)c1cccc(Cl)c1', 'C[C@@H]([NH2+][C@H](CO)c1ccccc1Cl)c1cccc(F)c1', 'C[C@@H]([NH2+][C@@H]1CCN(CCn2cc[nH+]c2C(C)(C)C)C1)c1nccs1', 'O=C(/N=C\\N=c1\\scc(-c2ccc([N+](=O)[O-])cc2)[nH]1)c1ccc[nH]1', 'O=C(/N=C\\c1ccc(Cl)cc1)N1CCc2cc(Cl)cc([N+](=O)[O-])c2C1', 'Oc1cccc(OCc2cccc3ccccc23)c1', 'Oc1cccc(O)c1C(=O)N1CCC2(CC2)OCCN1Cc1ccccc1', 'Oc1cccc(Oc2ccccc2NCc2ccc(C(F)(F)F)cc2)c1', 'Oc1cccc(O[C@@H]2CCOc3ccc(F)cc32)c1', 'Oc1cccc(O[C@H]2CCN(c3ncccn3)c3ccccc32)c1', 'NS(=O)(=O)N(CC(=O)Nc1ccc(F)c(Cl)c1)[C@@H]1CCC[NH+](C2CC2)C1', 'OC[C@@H]1CN(Cc2cccc([S@@](=O)C(F)F)c2)CCS1', 'OC[C@@H]1c2ccc(F)cc2CCN1C(=O)CCCCn1cccn1', 'OC[C@@H]1N=N/C(=N\\CCCc2ccco2)C1', 'OC[C@@H]1SC(N2CCCC2)=NC(=O)C[C@H]1[C@@H]2CCCN(C[C@H](O)c1ccccc1)CS2', 'OC[C@@H]1[C@H](OCc2ccccc2)c2ccc(F)cc2O1', 'Clc1ccc(CC[NH2+]Cc2ccccc2)cc1', 'Clc1ccc(C(=O)COC[C@@H]2CCOC2)cc1', 'Clc1ccc(Cc2n[nH]c(-c3ccco3)n2)cc1', 'Clc1ccc(C2CC2)cc1', 'Clc1ccc(CO/N=C/CCc2ccc3c(c2)OCO3)c(Br)c1', 'Clc1ccc(CN2CCN(C3CC3)CC2)cc1', 'Clc1ccc(C[NH+]2C[C@@H]3CCC[C@@H]3C2)cc1[N+](=O)[O-]', 'Clc1ccc(C[NH2+][C@@H]2CCO[C@]2(c2ccccc2)c2ccco2)cc1', 'Clc1ccccc1-c1nnc(C[NH+]2CCN(Sc3ccccc3)CC2)c(=O)[nH]1', 'Clc1nnnn1CC(=O)N(Cc1ccccc1)[C@H]1CCS(=O)(=O)C1', 'Clc1nnnn1Cc1cc(Cl)cc(Cl)c1', 'Clc1nnnn1C1CCN(C(=O)c2cc(Cl)c3c(c2)OCCO3)CC1', 'Clc1nncnc1C(=O)Nc1ccc(F)cc1', 'Clc1nncnc1C[NH+]1CCOCC1', 'Clc1nncnc1[N-]Sc1ccncc1', 'Clc1nncnc1Nc1ccc(-c2ccccc2)cc1', 'Clc1nncnc1/C=C/C(=O)Oc1ccc(Cl)cc1', 'Clc1nncnc1[NH2+]C[C@@H]1C=C[C@H](C(=O)[O-])C1', 'N[C@@H]1c2ccccc2C[NH+]1C/C([O-])F', 'CN=C/[C@@H]1c2ccccc2C(=O)N1c1ncccn1', 'CN=C(C#N)CC1(c2ccc(OC)cc2)CC1', 'CN=C1/C(=N/C(=O)[C@@H](C#N)c2ccc(Cl)cc2)S/C1=N\\c1cccc(F)c1', 'O=[N+]([O-])c1ccncc1/C=N/N=C[C@H]1C=C[C@@H](O)C1', 'ClC[C@@H]1CNC(=O)c2ccccc21', 'ClC[C@@H]1CN(c2ncccn2)C[C@H]1c1ccccc1F', 'ClC[C@@H]1CNc2ccc(F)cc21', 'ClC[C@@H]1CN2CCCC[C@@H]2CO1', 'Oc1cc(COC[C@@H]2COc3ccccc3O2)c(Cl)cc1Cl', 'Oc1cc(COc2ccccc2C=O)c2ccccc2c1O', 'Oc1cc(Br)c(Cn2ccnn2)c(Cl)c1', 'Oc1cc(Br)c(OCc2cc(F)cc(F)c2)c(O)c1', 'Oc1cc(Br)c(N/N=C\\c2ccccc2)cc1Cn1ccnc1', 'Oc1cc(Br)c([C@@H]2C[NH2+]CCC[C@H]2O)cc1F', 'Oc1cc(Br)c(-c2ccncc2)c(Br)c1', 'Oc1cc(Br)c(Cl)cc1NCC[C@H]1CCCCCC[NH2+]1', 'Oc1cc(Br)c([O-])c(C#CCOc2ccccc2)c1', 'Oc1cc(Br)c([C@H]2NN3[C@H](CC2)OC3C(=O)C2=CCCCC2)cc1Cl', 'Oc1cc(Br)c(Br)cc1OC(F)F', 'C=CCn1c(=O)c(C)nnc1SCCC(=O)Nc1ccc(C#N)cc1', 'C=C1N=C(C(=O)Nc2nncs2)[NH+](Cc2ccc(OC)cc2)C1=O', 'C=C1NC(=[NH2+])[C@@H]1/C=N/NC(=O)C(=O)N/N=C/c1ccc(C)cc1', 'C=C1NN(C(=O)c2cnccc2)[C@H]2N=CN=C2[C@H]1C', 'C=C1Nc2ccc(C#N)cc2[C@H]1c1cccc(Br)c1OC(F)F', 'Clc1cc(Br)c(CNc2ccc3c(c2)CCCO3)cc1Cl', 'Clc1cc(Br)cnc1COc1ccc(Cl)cc1Cl', 'Clc1cc(Br)cc(/C=N/N2CCOCC2)c1', 'Clc1cc(Br)c2ccccc2c1Cl', 'Clc1cc(F)c(Cl)cc1Oc1ccccc1F', 'Clc1cc(/N=C/CSc2cc(Br)c3ccccc3n2)ccc1O', 'Clc1cc(F)c(C(F)(F)F)cc1OC1CCC1', 'Clc1cc(F)cc(CN2CC[NH+](CCOc3ccccc3)CC2)c1', 'Clc1cc(Cl)c(N2CCN([C@@H]3[C@@H](O)NC2=O)CC3)cc1Cl', 'Clc1cc(Cl)cnc1-c1ccc2c(c1)COCO2', 'Clc1cc(Cl)ccc1Cn1cc(C(F)(F)F)ccc1=O', 'Clc1cc(Cl)c2ccccc2c1C[NH+](Cc1ccccn1)C1CCCCC1', 'Clc1ccsc1SCC1(Cn2ccnc2)CCOCC1', 'Clc1ccsc1S(=O)(=O)N1CCc2nccnc2C1', 'Clc1ccsc1Sc1ccccc1OC(F)F', 'Clc1cccc(CCOCCCOc2ccccc2F)n1', 'Clc1cccc([S@@](=O)c2ccc(Br)o2)c1', 'Clc1cccc(N2CC[C@@H](C(=O)[O-])C2)c1', 'Clc1cccc(F)c1N1CCOc2cccc(F)c21', 'Clc1cccc([C@@H]2CCCN2C[C@@H](O)COc2cccc(Cl)c2)c1', 'Clc1cccc(-c2nnc([C@H]3CCCN(S(=O)(=O)CC4CCCCCC4)C3)o2)c1', 'Clc1cccc(Sc2nnc(NC[C@@H]3CCCO3)s2)c1', 'Clc1cccc(Br)c1OCCc1ccccc1', 'Clc1cccc(/C=N/Nc2c(Cl)cccc2Cl)c1', 'ClC[C@H](CC1CCN(C(=O)c2cccnc2)CC1)n1cccn1', 'ClC[C@H](C(=O)NCc1ccc([N+](=O)[O-])cc1)N1CCCSCCC1', 'ClC[C@H](Cc1ccsc1)Nc1ccc(Br)cc1[N+](=O)[O-]', 'ClC[C@H](CO)Nc1ccc(C#Cc2ccccc2)cc1', 'ClC[C@H](CN1CCN(c2nc3ccccc3nc2C(F)(F)F)CC1)N1CC[NH+](CCCO)CC1', 'ClC[C@H](C[C@@H]1CCCO1)c1ccc(Br)s1', 'ClC[C@H](CS(=O)(=O)c1cc(F)ccc1F)Cn1cccn1', 'ClC[C@H](C[NH+]1CCc2cc(Cl)ccc2N1C[C@@H]1CCCO1)N1CCCC1', 'NS(=O)(=O)CCCNC(=O)Nn1cccc1', 'NS(=O)(=O)CC(=O)N(CC(=O)N1CCCCC1)c1cccc(Cl)c1', 'NS(=O)(=O)CCc1nc(-c2ccccn2)no1', 'NS(=O)(=O)CCN(C(=O)Cc1ccco1)Cc1ccccc1Br', 'NS(=O)(=O)CC[C@@H](NC1CC1)c1ccccc1F', 'NS(=O)(=O)CC[C@H](c1ccccc1)c1n[nH]cc1C(C)=O', 'NS(=O)(=O)CC[NH+]1CCc2ccccc2C1', 'NS(=O)(=O)CC/C=C(\\C=C\\C=C/c1ccc(Br)cc1)NC(=O)c1ccccc1Cl', 'Nc1c(Oc2ccc(C(N)=O)nc2CO)nc2ccccc2c1Cl', 'Nc1c(O[C@@H]2CCCc3ccccc32)ncnc1NC(=O)[C@@H]1C[C@@H]2c3ccccc3C(=O)N12', 'Nc1cnn(CCn2cc(Cl)c(=O)n(-c3ccccc3)c2=O)c1O', 'Nc1cnnn1Cc1cc(Br)ccc1O[C@H]1CCOC1=O', 'N#C[C@H](C(=O)Cc1cccc(Cl)c1)CCS(=O)(=O)c1ccccc1', 'N#C[C@H](c1ccccc1)[C@H]1CCC[NH+](Cc2ccc(C(C)(C)C)o2)C1', 'N#C[C@H](NCc1ccc(N2CCOCC2)c(Cl)c1)C1(c2cc(Cl)ccc2Cl)CCCC1', 'N#C[C@H](F)c1ccccc1OCc1ccc(Br)cc1', 'N#C[C@H]([C@@H]1O[C@@H](O)[C@H]1CC(N)=O)[C@@H]1CCCC[C@H]1C(=O)[O-]', 'O=C(c1c[nH]nc1-c1ccc2c(c1)OCO2)N(C1CCCCC1)S(=O)(=O)c1ccccc1', 'O=C(c1c[nH]nc1N1Cc2ccccc2O[C@H](c2ccc(Cl)cc2)C1)N1CCC(c2nccs2)CC1', 'O=[N+]([O-])c1cccc2c1[nH]c(=S)n2CCS(C)(=O)=O', 'O=[N+]([O-])c1ccc2oc([C@H]3CCC[NH2+][C@@H]3Cc3ccc(F)cc3)cc2c1', 'O=[N+]([O-])c1ccc2[nH]c(C[NH2+]Cc3ccnc(F)c3)nc2c1', 'O=[N+]([O-])c1ccc2c(c1)[C@@H]1C=CCC[C@H]1[C@H](c1ccc(F)cc1)N2C1CCCCC1', 'O=[N+]([O-])c1ccc2nc(-c3cccc([N+](=O)[O-])c3)[nH]c(=O)c2c1', 'N[C@@H](O)CCN1CCCCC1=O', 'NS(=O)(=O)/C(=O)c1cccc2c1[C@H]([C@@H]1CCCCN1C(=O)c1ccco1)N2', 'NS(=O)(=O)/C(=C(\\C)c1csc(NC(=O)c2ccc(F)cc2)n1)c1ccccc1', 'NS(=O)(=O)/C(=N/Nc1ncccc1Cl)c1cc(F)ccc1O', 'Clc1ccsc1F', 'Clc1ccsc1/C([O-])=N/Oc1cncc(Cl)c1', 'Clc1ccsc1/N=C/c1cccc(COc2cccnc2)c1', 'O=C(Cc1ccc(Cl)cc1)NC[C@H]1CCCN(c2ncnc3ccccc23)C1', 'O=C(Cc1noc2c1CCCC2)Nc1ccc(Br)cc1', 'O=C([C@H](Nc1ccc2c(c1)OCCO2)c1ccccc1)N(C1CCCCC1)[C@H]1CS(=O)(=O)CCS1', 'O=C([C@H](N[C@@H]1CCc2ccccc21)[NH+]1CCCC1)c1ccc2c(c1)OCCO2', 'O=C([C@H](NS(=O)(=O)c1cccc2ccccc12)[NH+]1CCCC1)c1ccc(F)cc1Cl', 'O=C(CSCC(F)(F)F)NNC(=O)c1ccc(S(=O)(=O)N2CCCC2)cc1', 'O=C(CSC(=O)Nc1ccc2c(c1)nc1ccccn12)N1CCC(Cc2ccccc2)CC1', 'O=C(CSCc1ccco1)N/N=C/c1ccc(Oc2cc([N+](=O)[O-])ccc2Cl)cc1', 'O=C(CSCN1C(=O)[C@H]2CCN(Cc3ccccc3)C2=C1CN1CCOCC1)c1ccccc1', 'O=C(/C=C1/S/C(=C/c2cccc(F)c2)N1c1cccnc1)N1CCCC[C@@H]1CNc1ccc2c(c1)CCCO2', 'O=C(/C=C1CCCCC1)Nc1ccccc1N(Cc1ccccc1)C1CCCCC1', 'O=C([O-])[C@@H](Cc1ccc(F)cc1)N1CCCCC1', 'O=C([O-])[C@@H](c1ccccc1)[C@H]1CCCC[NH+]1Cc1ccc(Cl)cc1', 'O=C([O-])[C@@H](Oc1ccccc1Br)n1cccn1', 'O=C([O-])[C@@H]([NH3+])c1csc(Cc2cccc(F)c2)n1', 'Nc1nnc(C2=NN2c2cccc(C(F)(F)F)c2)n1/N=C/[C@H]1CCCC[C@@H]1C(=O)Nc1cccc2[nH]ccc12', 'Nc1nncn1CCc1ccccc1', 'N[C@@H](O)C[NH+]1CCC([C@H]2CCOC2)c2ccccc21', 'N[C@@H](O)c1csc(N2CCCC2)n1', 'N[C@@H](O)[C@@H]1CCCC[C@H]1[NH3+]', 'Clc1nnnn1/C(=C\\c1nc[nH]n1)c1ccc(I)cc1', 'Clc1nnnn1/N=C/c1ccc([N+]2=C(CCCc3ccccc3)C(=O)NC2=S)cc1', 'COc1nc(C)c(C(=O)O[C@H](C)C(=O)N2CCN(c3ccc(C)cc3)CC2)cn1', 'COc1nc(NC(=O)Cc2c[nH]c(C)c2C)cc(NCc2ccccc2)c1C', 'COc1nc(F)ccc1CC(=O)O[P@](=O)(=O)Cc1ccccc1', 'COc1nc(-c2cccc(OC)c2)cc(N2CCCC2)c1[N+](=O)[O-]', 'COc1nc(S(=O)(=O)CC(C)C)ccc1C', 'COc1nn[nH]c1C(=O)NC[C@@H](CCO)c1ccccc1', 'COc1nn[nH]c1/C=C/C(=O)N[C@@H](CO)c1ccccc1', 'CNS(=O)(=O)N1CCN(C(=O)[C@@H](C)Cc2c(C)noc2C)CC1', 'COc1ncnc(NC(=O)[C@@H](C)n2c(=O)n(Cc3ccc(F)cc3)c(=O)c3ccccc32)c1C', 'COc1ncncc1CNCc1cc(F)ccc1F', 'COc1nn(CC)c(CN2CC(=O)N(c3ccc(C)cc3C)C[C@@H]2c2ccccc2)c1C', 'COc1nn(C)cc1C(=O)N1CC[C@H](C#N)C[C@@H]1c1ccc(C)o1', 'Oc1ccc([N+](=O)Cc2coc3ccccc23)cc1OC(F)F', 'Oc1ccc([N+](=O)C2CCCC2)cc1OC[C@@H]1CCC[C@@H]1O', 'Oc1ccc([N+](=O)C=O)cc1Cl', 'Oc1ccc([N+](=O)COc2ccccc2Cl)cc1O', 'Oc1ccc([N+](=O)CN2CCN(c3ncccn3)CC2)cc1Cl', 'Oc1ccc([N+](=O)CCl)cc1N1CCCC1', 'CCn1c(=S)[nH]c2ccccc21', 'CCn1nc(C)cc1CN1C[C@H]2CC[C@@H]1CC(CC(F)F)C2', 'O[C@@H]1[C@@H](OCC(=O)OC2CCCCC2)[C@@H]2CCCC[C@@H]21', 'Clc1ccsc1[C@@H](Cl)CCOc1ccccc1F', 'Clc1ccsc1[C@@H]1Oc2c(nnc3c2SCC3)C1', 'O=[N+]([O-])c1c([O-])cnn1-c1ccccc1', 'O=[N+]([O-])c1c([O-])nsc1Nc1cccc(I)c1', 'O=[N+]([O-])c1c([O-])sc2nc([O-])[nH]c12', 'O=[N+]([O-])c1c([O-])[nH]c(=O)n1CC(=O)Nc1ccc(Oc2ncccn2)cc1', 'COc1ncncc1C(=O)Nc1cc(S(=O)(=O)C(=O)N2CCCC2)ccc1F', 'COc1ncnc(CNc2cccc(OC(C)C)c2)c1[N+](=O)[O-]', 'COc1ncnc(Br)c1C(=O)N1CCC(OC)CC1', 'COc1ncnc(Sc2nn3c(=O)c(C)c(C)nc3c(=O)c3nc(C)sc23)c1N', 'COc1ncnc(NCCc2ccccc2)c1Br', 'CC[S@](=O)[C@H]1CCCC[C@H]1NC(=O)C(=O)Nc1ccc2[nH]cnc2c1', 'Clc1nnnc(Cl)c1-n1cccn1', 'Oc1cccc2cc(-c3cccc(Cl)c3)[nH]c12', 'Oc1cccc2cc(OC[C@H](O)COc3ccccc3)[nH]c(=O)c12', 'Oc1cccc2cc(O[C@H]3CCOC3)cc(C(F)(F)F)c12', 'Oc1cccc2cc(O)ccc12', 'Clc1cc(/C=N/NC(=S)NC2CC2)nc2cccnc12', 'Clc1cc(NSc2ccc(Cl)s2)ccc1C(F)(F)F', 'Clc1nnn(Cc2ccn[nH]2)n1', 'Clc1cc(/N=C\\c2ccc([N+](=O)[O-])cc2F)cs1', 'Cc1sc(C)c([C@H](C)NC(=O)c2ccc(C#N)cc2)c1C', 'Cc1sc(=O)n(CCC(=O)N[C@H]2CCSC2)c1C', 'Cc1sc(NC(=O)C(C)(C)C)cc1C(=O)N(C)CCC(F)(F)F', 'COc1nnn(-c2cc(N)ccc2Sc2ncccn2)c1C', 'COc1nnn(CCCNC(=O)[C@H]2CCO[C@H]2C)c1[N+](=O)[O-]', 'Clc1ccc([C@@H](CCCl)Nc2ccccc2F)cc1', 'Clc1ccc([C@@H](O)CCS(=O)(=O)c2ccc(Br)cc2)cc1', 'Clc1ccc([C@@H](N[C@@H](C#N)c2ccccc2)C(F)(F)F)cc1', 'Clc1ccc([C@@H]([C@@H]2CCOC3(CCCCC3)C2)OC[C@H](O)CO)cc1', 'Clc1ccc([C@@H](Cl)Cc2ccc(F)cc2)cc1Cl', 'Clc1ccc([C@@H]([O-])C[C@H](O)Cc2cc(Cl)ccc2F)cc1', 'Clc1ccc([C@@H](Br)Cc2ccccc2)cc1', 'Clc1ccc([C@@H]([NH2+][C@H]2CCc3ccc(F)cc32)[S@@](=O)Cc2ccccc2)nc1', 'OCS(=O)(=O)c1ccc(F)cc1Oc1ccc(C(F)(F)F)cc1', 'OCSCCCN1CC[NH+](C2CCCCC2)CC1', 'OCSCc1nc2cc(Cl)ccc2nc1Cl', 'OCSC1=N[C@@H](COc2cccc(Cl)c2)N(C(=O)c2ccc([N+](=O)[O-])cc2)C1', 'OCSC[C@@H](CO)[C@@H]1C[NH2+]CCO1', 'OCSC[C@H]1CCCC[C@@H]1[NH3+]', 'Clc1nnnc(SCn2ncc3ccc4c(c23)CCCCCC4)c1', 'Clc1nnnc(S(=O)(=O)N2CCOCC2)c1-n1cccc1', 'Clc1nnnc(Sc2ccnc(N3CCOCC3)c2)n1', 'Clc1nnnc(S[C@@H]2CCCOc3ccc(OC(F)(F)F)cc32)c1Br', 'Clc1nnnc(S[C@H]2CCO[C@@]3(CCC[C@@H]3c3ccccc3)C2)n1', 'O=C(c1cn2ccccc2n1)N1CC[C@@H](CO)[C@H]1NC(=O)c1ccccc1Cl', 'O=C(C1CCN(c2ncccc2Cl)CC1)N1CC=C(c2ccccc2)CC1', 'O=C(COc1cccc2ccccc12)Nc1ccc(N2CCOCC2)cc1', 'O=C(COCC(=O)N1CCOCCC1)Nc1ccc2[nH]c(=O)[nH]c2c1', 'O=C(COCc1ccccc1)Nc1cccc(OCc2ccccc2)c1', 'C[C@@H]([C@@H](O)c1ccccc1)N1CCc2ncc(C(F)(F)F)cc2C1', 'Cc1ccsc1/C=C/C(=O)N1CCC(C(=O)[O-])CC1', 'Cc1cc(CC(=O)NCc2nc3ccccc3[nH]2)c2ccccc2n1', 'Cc1ccc(C(=O)CC[n+]2cccc[nH+]2)cc1F', 'CNc1ccc(Cl)c(C)c1C(=O)N(C)[C@@]1(C)CCCC[C@@H]1C', 'C[C@@H](c1ccccc1)N(C)C(=O)Cc1c[nH]c2ncccc12', 'C[C@@H](c1cnn(C)c1)c1ccccc1C[NH+](C)Cc1cnc[nH]1', 'Oc1ccc(/C=N/Nc2nnnn2-c2ccccc2)cc1', 'Oc1ccc(/C=N\\CCCCSc2ccccc2)cc1', 'Oc1ccc(F)c(C[C@@H]2CCC[NH2+]2)c1', 'Oc1ccc(F)c(Oc2cc(F)ccc2F)n1', 'Oc1ccc(F)c(F)c1NCCC(F)(F)F', 'Oc1ccc(F)c([C@@H]2CCCN2c2cccc(C(F)(F)F)c2)c1', 'Oc1ccc(F)c(-c2nnc(NC[C@@H]3CCCO3)s2)c1', 'Oc1ccc(F)c(SCC(=O)N2CCO[C@@H](c3ccc(Cl)cc3)C2)c1', 'Oc1ccc(F)c(Cl)c1Cl', 'Oc1ccc(F)c(/N=C\\Nc2ccccc2F)c1NS(=O)(=O)c1ccccc1', 'Oc1ccccc1-c1ccccc1F', 'Oc1ccccc1-n1cc(/N=C/c2cccs2)c2ccccc21', 'Oc1ccccc1/C=N/N(Cc1c(F)cccc1Cl)[C@@H]1CCS(=O)(=O)C1', 'Oc1ccccc1/N=C(\\N/C1=N/S=C1N1CCC[C@@H]1c1ccc(Cl)cc1)c1ccccc1', 'Oc1ccc(/C=C/C(=O)N2CC=C(c3ccccc3)OC2=O)cc1Cl', 'Oc1ccc(/C=C(/CO)C[NH+](C)CCCS)cc1NC(=O)c1ccccc1', 'Oc1ccc(/C=C2\\SC(=S)N(CC(=O)NCc3ccc(F)cc3)N2Cc2ccccc2)cc1', 'Oc1cc(C=Nc2ccccc2)cc(I)c1O', 'Oc1cc(C=O)c2sccc2c(=O)c1Br', 'O[C@@H]1c2ccccc2C[C@@H]1CC[NH+]1CCCC[C@H]1C1OCCO1', 'O=C(c1ccc(F)c(F)c1)N1CCSC[C@@H]1C(=O)[O-]', 'O=C(c1ccccc1)c1ccc(Br)cc1OCC[NH+]1CCCC1', 'O=C(c1ccco1)N1CC(Cn2cc[nH+]c2)c2ccccc21', 'O=C(c1cccnc1)N1CCC[C@H](c2nccn2Cc2ccccc2)C1', 'O=C(c1cccs1)c1ccc(N2CCN(c3cccc(Cl)c3)CC2)cc1', 'O=C(c1ccc[nH]1)N1CC[C@@H](c2ccccc2)[C@@H]1C#CCc1ccccc1', 'Oc1cccc(Br)c1Cl', 'Oc1cccc(NC(=O)CCN2C(=O)[C@H](COc3cccc(Br)c3)C2=O)c1', 'Oc1cccc(Nc2cccc(F)c2Br)c1O', 'Oc1cccc(N2C(=O)[C@H]3CN(c4ccc(Br)cc4Cl)CN3C2)c1', 'Oc1ccccc1[C@H](O)[C@@H]([C@H]2CCOC2)N1CCN(c2ccccc2F)C1', 'Oc1ccccc1[C@H]1C2C[C@@H]3CCCCC[C@H]3C[C@@H]21', 'COc1nccc([C@@H]2N=NC(=O)c3ccc(OC(C)C)cc32)c1C', 'COc1ncccc1F', 'Cc1sc(NCCN(C)C)nc1S(=O)(=O)N1CCN(c2ccc(F)cc2)CC1', 'Cc1sc(N(CC[NH3+])Cc2ccccc2)nc1C', 'Cc1sc(N)c([C@@H]2CC[NH+](Cc3cccc(F)c3)C2)n1', 'NS(=O)(=O)N[C@@H](c1ccccc1Cl)C1([NH+]2CCOCC2)CCCC1', 'NS(=O)(=O)N[C@@H]1CCCN(CC(F)(F)F)C1', 'O=[N+]([O-])c1cc(SC[C@H]2CC=CCC2)ccc1O', 'O=[N+]([O-])c1cc(S(=O)(=O)CCCc2cn3ccccc3n2)ccc1F', 'O=[N+]([O-])c1ccn([C@H](CO/N=C\\c2c[nH]c3ccccc23)N2CCOCC2)c1', 'O=[N+]([O-])c1ccn([C@H]2CCCCN2C2CC[NH2+]CC2)n1', 'COc1nn[nH]c1CNC(=O)NCc1ccc[nH+]c1N(C)C', 'COc1nn[nH]c1CC(=O)N1CC[C@@H](C(=O)[O-])[C@@H]1C', 'COc1nn[nH]c1C(=O)Nc1ccccc1S(=O)(=O)c1ccc(F)cc1', 'O=C([C@H](c1ccc(Cl)cc1Cl)N1CC[NH+](C2CCCCC2)C1)N1CCOCC1', 'CN[C@H]1C[C@@H](C)N=C(C)[C@H]1O[C@H]1C[C@H](C)[C@H](O)C1', 'CN[C@H]1c2ccc(OC)c(C)c2C(=O)N(CC(C)C)C1', 'CN[C@H]1Nn2c(-c3ccc(F)cc3)cnc2N=C[C@@H]1SC', 'CN[C@H]1[C@@H](OCc2ccccc2)N(Cc2ccccc2F)C1=O', 'CN[C@H]1[C@H](OCC(C)C)CCCc2cccc(Br)c21', 'Oc1ccc([C@H]2CCOCC2)cc1', 'Oc1ccc([C@H]2OC[C@@H](O)C[C@@H]2C[NH2+]Cc2cccc(Br)c2)cc1', 'Oc1ccc([C@H]2NC(=O)N/C(=C\\c3cccc(O)c3)N2Cc2ccccc2)cc1', 'Oc1ccc([C@H]2[NH2+]C[C@H](Nc3ccc(S(=O)(=O)N4CCOCC4)cc3)CSC2=O)cc1F', 'Nc1nc2sc3ccccc3n2c1NCc1cccnc1', 'Nc1nc2c(n1Cc1ccc(C(F)(F)F)cc1)CCC2', 'Nc1nc2ncc([N-]S(=O)(=O)c3ccc(Cl)cc3)cc2cc1C[NH2+]C1CC1', 'NS(=O)(=O)/N=C/c1ccc(-c2ccc(Cl)cc2)cc1', 'NS(=O)(=O)/N=C(\\NCc1ccccc1)c1ccc(-c2ccc(Cl)cc2)cc1', 'NS(=O)(=O)/N=C\\c1ccccc1Br', 'NS(=O)(=O)/N=C(\\[O-])CCS(=O)(=O)c1ccccc1N(C)C', 'NS(=O)(=O)/N=C(/NCc1cccc(Cl)c1)n1nc(-c2cccs2)c2ccccc21', 'NS(=O)(=O)/C=C/c1cccnc1', 'NS(=O)(=O)/C=C(\\CC#N)C(=O)Nc1ccc2c(c1)OCO2', 'NS(=O)(=O)/C=C\\c1c(Cl)cccc1Cl', 'NS(=O)(=O)Cc1cc(Cl)ccc1-c1nc(-c2ccncc2)cs1', 'NS(=O)(=O)Cc1nccc(CO)c1C(C)C', 'NS(=O)(=O)NCCOC1(c2ccccc2)CCC1', 'NS(=O)(=O)NCc1csc(C(=O)N2CCCC[C@@H]2C)n1', 'NS(=O)(=O)NC1CCC2(CC1)OCCO2', 'NS(=O)(=O)NC[C@@H](c1ccco1)N1CCN(C(=O)[C@@H]2CCc3ccccc3C2)CC1', 'NS(=O)(=O)NC[C@H](c1ccc(Cl)cc1)N1C(=O)COc2ccccc21', 'NS(=O)(=O)NC[C@](O)(c1cccs1)C(Cl)(Cl)Cl', 'NS(=O)(=O)NC[C@@]12CCN(c3ccccc3)[C@@H]1Cc1ccccc1-2', 'Nc1[nH]nc2c1CCCC2', 'Nc1[nH]nc2nc(COc3ccc(F)c(Cl)c3)[nH]c12', 'Nc1[nH]ncc1CN(C1CCCCC1)[C@H]1CCSC1', 'Nc1[nH]ncc1Nc1ncnc(Nc2ccc(Cl)cc2)n1', 'Nc1[nH]ncc1[C@@H]([N+](=O)[O-])C(F)(F)F', 'Nc1[nH]ncc1-c1nnc([C@H]2CCCN(C(=O)CCc3ccc(Cl)cc3)C2)n1Cc1ccccc1', 'Nc1[nH]ncc1S(=O)(=O)CCC(F)F', 'N#Cc1coc(S(=O)(=O)Nc2c(-c3ccccc3)cnn2-c2ccc(F)cc2)c1', 'N#Cc1csc(NCc2cnc3ccccn23)c1', 'N#Cc1c(F)cccc1NC(=O)N[C@H](CO)Cc1ccccc1', 'N#Cc1cccc(S(=O)(=O)NC2CC2)c1', 'N#C[C@@H](c1ccccc1)N1CCC(C[NH2+]C[C@H]2CCOC2)CC1', 'N#C[C@@H](O)C[C@@H]1COCCN1Cc1ccc(Br)cc1', 'N#C[C@@H](Nc1ncccc1O)c1cccc(-n2cccn2)c1', 'N#C[C@@H]([C@@H]1CCCON1c1ccccc1)c1ccc(-n2nnnc2CS)cc1', 'N#C[C@@H](Sc1cnccn1)Cc1cc(F)cc(F)c1', 'NS(=O)(=O)C[C@H](NC1=NS(=O)(=O)c2ccccc21)c1ccccc1', 'NS(=O)(=O)C[C@H]1C[C@@H]1c1ccccc1', 'N[C@@H]1N=Nc2nnnn21', 'N[C@@H]1NC(=O)/C1=C/c1ccc(-c2ccc(F)cc2Cl)o1', 'Clc1cc(Br)c2ncnn2c1Cl', 'Clc1cc(NC(c2ccc3c(c2)OCCO3)[NH+]2CCCCCC2)ccc1Cl', 'Clc1cc(NC2(n3cccc3)SCCN2)c2ncccc2c1', 'Clc1cc(NC[C@@H](n2cncn2)[NH+]2CCCC2)ccc1F', 'Clc1cc(NC[C@@]2(O)CCSC2)ccn1', 'O=[N+]([O-])c1cc(Br)cc(SCc2ccccc2)c1', 'Nc1n[nH]c(-c2ccc(F)cc2)c1C(=O)Nc1ccc2c(c1)COC2', 'Nc1n[nH]c2cc([N+](=O)[O-])ccc12', 'Nc1nnn([C@H](CCO)C2CC2)n1', 'Nc1nnnn1CC[S@@](=O)c1ccccc1Cl', 'NS(=O)(=O)/C=c1\\nc(Cl)nc(Nc2ccccc2)c1=O', 'O[C@@H](Sc1nncn1Cc1c[nH]nc1c1ccccc1)C1CCOCC1', 'NS(=O)(=O)/N=c1/c(=N/C(=O)Cc2ccccc2)c(C2CC2)nn1-c1ccccc1', 'N=Cc1ccc([C@@H](C)NC(=O)C2CC2)cc1', 'N=Cc1nc(-c2ccccc2)cs1', 'N=Cc1c([N+](=O)[O-])cccc1C(=O)NC[C@@H](O)C(=O)c1ccccc1', 'N=Cc1ccc(O[C@H](C)C(=O)N2c3ccccc3C[C@H]2c2ccccc2)cc1', 'N=Cc1cnc(Cl)nc1-c1ccc(Cl)cc1', 'N=Cc1nonc1CSCc1ccccc1', 'N=Cc1n[nH]c(C(=O)N2CC[C@@H](C)[C@@H](O)C2)n1', 'N=Cc1nc(-c2ccccc2)nc2ccccc12', 'N=C1/C(=C\\C=C\\c2cccc(Cl)c2)C[NH2+]C1', 'N=C1/N[N+](=O)OC1', 'O=C(C1=C([O-])C(=O)N(C[C@@H]2CCCO2)[C@@H]1c1cccc(Br)c1)c1ccc2c(c1)OCO2', 'O=C(C1=N[C@@H]2[C@H](c3ccccc3)C[C@@](C)(C)[C@H]12)c1ccccc1', 'O=C(C1=c2ccccc2=[NH+][C@H]1N1CCOCC1)N1CCc2nccnc2C1', 'Oc1cc(-n2cn/cn2)ccc1-c1ccccc1', 'Oc1cc(-n2cnc3ccccc32)cc([N+](=O)[O-])c1', 'Oc1cc(-n2cnnn2)cc(-c2ccco2)c1', 'Oc1cc(-n2nnc(-c3ccc(Cl)cc3)n2)ccc1Oc1cccc(Cl)c1Cl', 'Oc1cc(-n2nncc2F)nc(-c2ccccc2Cl)n1', 'O=C([C@H]1CC[C@@H](C[NH3+])C1)[C@H](Cc1ccccc1)N1C(=O)c2ccccc2C1=O', 'O=C([C@H]1COCCN1c1c(F)cccc1F)N1CCN(C(=O)COc2ccccc2)CC1', 'O=C([C@H]1C[C@H]1c1cccs1)N1CCN(c2ccc(F)c(F)c2)CC1', 'O=C(/N=c1/oc2ccccc2c2ccccc12)c1ccc(NC(=O)C(F)(F)F)cc1', 'O=C(/N=c1/[nH]c2ccccc2c(=O)n1-c1ccccc1)c1csc(C2CC2)n1', 'NS(=O)(=O)Cc1ccccc1C(=O)N1CCC[C@@H]1C(N)=O', 'NS(=O)(=O)Cc1cn2ccccc2[nH+]1', 'O[C@@H](c1cc(Br)cnc1Cl)Cc1ccc(Cl)cc1F', 'O[C@@H](c1ccccc1Cl)[C@H]1CSCCSC1', 'Nc1cn[nH]c(=O)c1N', 'Nc1cn[nH]c1[C@@H]1CCCN1C(=O)C1CCCC1', 'Oc1cc(O)c(C(=O)Nc2ccc3[nH]nnc3c2)cc1Cl', 'Oc1cc(O)ccc1O[C@H]1CCCc2ccccc12', 'Oc1cc(O)c2ccc(Cl)cc2c1OC(F)c1ccccc1', 'Oc1cc(F)cc(C[NH+]2CCC[C@H]2c2cnoc2)c1', 'Oc1cc(F)ccc1Br', 'Oc1cc(F)cc2c1NC(=O)N2Cc1cc(Cl)sc1Cl', 'C=CCOCCCNC(=O)CNC(=O)c1cccc(C(F)(F)F)c1', 'C=CCOc1cc([N+](=O)[O-])ccc1OCc1ccc(OC)cc1', 'C=CCO[C@@H](C)C(=O)N[C@@H]1CC[C@@H]([NH3+])C1', 'OC[C@@]1(C(F)(F)F)c2ccccc2N=C(N2CCN(Cc3ccccc3)CC2)c1', 'OC[C@@]1(Cc2cccnc2)Oc2ccccc2C1', 'OC[C@@]1(COc2ccccc2C[NH2+]C(C)(C)C)CCCC1', 'OC[C@@]1(CNCc2cc3ccc(F)cc3n2C2CC2)CCCO1', 'OC[C@@]1(Cn2cccc2)CN(CCc2[nH+]ccn2Cc2ccc(F)cc2)C1', 'O=[N+]([O-])c1c(Br)sc(Br)c1/C=N/OCCn1ccc2ccccc21', 'O=[N+]([O-])c1c(Br)cccc1N1CCCCCCCCC1', 'ClCc1csc(COc2cccc(Br)n2)n1', 'ClCc1csc([N-]Cc2nc(-c3ccccc3)no2)n1', 'ClCc1csc(OCc2ccc(Cl)cc2Cl)n1', 'ClCc1csc(N2CCOCC2)n1', 'ClCc1csc(-c2ccccc2)n1', 'ClCc1csc(Sc2ccccc2)n1', 'ClCc1csc([C@H]2CCCO2)n1', 'ClCc1cn[nH]c1N1CCC[C@H]1c1ccccn1', 'ClCc1cc([N+](=O)[O-])ccc1I', 'ClCc1cc(C(F)(F)F)nc(N2CCOCC2)n1', 'ClCc1cc(OCc2cccc(Cl)c2)nn1CC1CC1', 'ClCc1cc(F)ccc1C1CC1', 'ClCc1cc(-n2cccc2)cc2c1NCCCO2', 'ClCc1cc(Cl)cc2c1OCOC2', 'ClCc1cc(Br)ccc1Oc1ccccc1', 'ClC[C@@H]1C[C@H](O)C[NH+]1Cc1nc2ccccc2c2ccccc12', 'ClC[C@@H]1C[C@H]1c1cc(I)c(Cl)cc1Cl', 'ClCc1nnc(-c2ccccc2)n1-c1cccc(Cl)c1', 'ClCc1nncn1CC(=O)N[C@H]1CCN([C@@H](c2ccccc2)c2ccccc2)C1', 'ClCc1nonc1Sc1ccccc1', 'ClCc1nonc1COC(c1ccccc1)c1ccccc1', 'ClCc1nnn(-c2ccccc2)n1', 'ClCc1nnn[n-]1', 'ClCc1nnn2c1N[C@@H](c1ccccc1-n1nnnc1COCCO)C(=O)CCC2', 'ClCc1nnnn1[C@@H]1CCC[C@@H](C2CC2)C1', 'Oc1cccc2cc(/C=N/N=C\\c3ccco3)oc12', 'Oc1cccc2cc([C@@H]3CCC[NH+]3C3CC3)ccc(=O)c12', 'ClC(c1ccccc1)c1cccc(Cn2cncn2)c1', 'ClC(c1ccoc1C(F)(F)F)[C@@H]1C[C@@H](I)C=CC[C@H]1C(=O)[O-]', 'ClC(c1cc(Cl)on1)c1cc(I)cc(O)c1', 'ClC(c1ccccc1Cl)[NH+]1CCC[C@H](n2cccn2)C1', 'ClC(c1cc2ccccc2oc1=O)c1cccnc1Sc1ccc(F)cc1', 'OC1CCC([NH2+]Cc2ccco2)CC1', 'OC1C[C@@H]2CC[C@@H]1C[NH+](Cc1ccccc1F)C2', 'C=CCc1ccccc1OCc1nnc([S@](=O)CC(C)(C)CC)n1C', 'COC[C@](C)(C#N)NC(=O)CC12CC3C[C@](C)(C1)C[C@@](C)(C2)C3', 'COC[C@]1(C)CCN(C(=O)c2cc(C)nn2C2CCOCC2)C1', 'O=C(Cc1c(F)cccc1Cl)NN[C@H]1CCS(=O)(=O)C1', 'O=C(Cc1ccccc1)NCc1nncn1C1CC1', 'O=C(Cc1coc(-c2ccc(Cl)cc2)n1)N1CCCC[C@@H]1c1ccccc1', 'O=C(Cc1cnn(-c2ccc3c(c2)OCO3)c1C#N)Nc1ccc2c(c1)OCO2', 'O=C(Cc1c[nH]c2ccccc12)N1CCN(C(=O)c2ccccc2)CC1', 'O=C([C@H]1[C@H](c2cccc(Br)c2)C[C@@H]1c1ccccc1)N1CCCN(CC(=O)[O-])CC1', 'O=C([C@H]1[C@H]2CCCCN2C1=O)N(c1ccc(-n2cnnc2)cc1)Cc1ccccc1', 'O=C(c1cnn(C[C@@H]2CCCO2)c1)N1CCN(c2ccc3c(c2)OCO3)CC1', 'O=C(c1cnnn1-c1ccccc1)N1CCC[C@H](c2ncccc2Br)C1', 'O=C(c1cnn2cc([N+](=O)[O-])cnc12)N(Cc1ccccn1)S(=O)(=O)c1ccc(F)cc1', 'O=C(C[NH+]1CCN(c2cccc(F)c2)CC1)Nc1ncc(-c2nccn2C(F)F)cn1', 'Oc1cc(C[NH2+][C@H]2CCOC3(OCCCC3)C2)ccn1', 'Oc1cc(C[NH2+][C@@H]2CCCC[C@H]2CCO)cc(Cl)c1O', 'O=C(c1c[nH]c2ccccc12)N(CCO)c1ccccc1', 'O=C(c1c[nH]c2nccc(Cl)c12)N1CCC(c2nc3ccccc3[nH]2)CC1', 'O=C(c1c[nH]c([N+](=O)[O-])c1)c1ccc(Br)s1', 'O=C(c1c[nH]c(=O)[nH]c1=O)N(Cc1ccccc1Cl)C(F)(F)F', 'O=C(c1c[nH]c(Nc2cc(Cl)ccc2F)c1)c1ccccc1', 'O=C(c1c[nH]c(-c2cccs2)n1)N(Cc1ccccc1)Cc1ccccn1', 'O=C(c1c[nH]c([O-])n1)N(Cc1ccccc1)C[C@H](c1ccccc1)C(=O)Nc1ccc2c(c1)OCCO2', 'O=C([O-])[C@H](Cc1ccc(/N=C/c2cccc(OC(F)F)c2)o1)c1ccccc1', 'O=C([O-])[C@H]1[C@H]2C=C[C@@]2(O)CCN1c1cccnc1', 'O=C(CN(Cc1ccccc1)c1ccccc1)NNc1cccc2ncccc12', 'O=C(CN(c1ccccc1)S(=O)(=O)/C=C/c1ccccc1)NCc1ccccc1', 'O=C(CNS(=O)(=O)c1ccccc1)NNC(=O)c1cc(-c2ccccc2)on1', 'O=C(CS(=O)(=O)c1ccccc1Cl)Nc1cc(Cl)cc(Br)c1', 'O=C(C[NH+]1CCC[C@H]1COc1ccc2n[nH]c(=O)cc2c1)N1CCCCC1', 'O=C(C[NH+]1CC(=O)N2CCC[C@@H]2C1=O)c1ccc(F)cc1', 'O=C(C[NH+]1CCc2sccc2C1)Nc1cccc(F)c1', 'O=C(C[NH+]1CCOCC1)NCCCc1ccc(F)cc1Cl', 'O=C(C[NH+]1CCN(Cc2cccnc2)C1)Nc1cnccn1', 'O=C(C[NH+]1CC[C@@H](c2nc3ccccc3s2)C1)NCCOc1ccc(N2C(=O)NC(=O)c3ccccc32)cc1', 'N#C/C(=C\\c1cccc(NC(=O)c2cnn(C3CCC3)c2)c1)C(N)=O', 'N#C/C(=C\\c1cc(Cl)ccc1OCCO)C(F)(F)F', 'N#C/C(=C\\c1cnc(-c2ccco2)cc1O)[C@@H]1CCCO1', 'N#C/C([O-])=C(\\CC(=O)c1ccc(F)cc1)c1cc(C#CCCO)ccc1O', 'N[C@@H](c1c(F)cccc1F)[C@H]1CCO[C@]2(CCSC2)C1', 'Clc1nncc(NC[C@@H]2COc3c(F)cccc32)n1', 'Clc1nncc(Nc2cccc(C(=O)[O-])c2)n1', 'Clc1nncc(N2CCSC[C@H]2Br)n1', 'Clc1nncc(N[C@@H]2CCC[C@H](O)C2)n1', 'Clc1nncc(N[C@H]2CCCC[C@@H]2n2cnnc2)c1Cl', 'Clc1nncc(N[NH+]2CCC[C@@H](C(=O)N3CCCCC3)C2)c1Cl', 'O[C@@H]1[C@@H](CC(=O)N2CCN(Cc3ccccc3F)CC2)CC(=O)N1C[C@@H]1CCCCO1', 'O[C@@H]1[C@@H](c2ccccc2)Oc2ccccc21', 'O[C@@H]1[C@@H](O)[C@@H](O)[C@@H]1Cc1cc(Cl)ccc1Cl', 'O[C@@H]1[C@@H](NS(=O)(=O)c2ccc3c(c2)C(=O)c2ccccc2S3)N=C2CCCC[C@@H]21', 'O[C@@H]1[C@@H]([NH2+]CC2CCCCC2)CSCCN1C[C@H](O)C(F)(F)F', 'CCOc1ccc2[nH]c3ccc(CC(=O)N4CCN(C(C)=O)CC4)cc3c2c1', 'CC[S@](=O)[C@H]1CCCC[C@@H]1NC(=O)Nc1ccn(Cc2ccc(Cl)cc2)n1', 'Cc1cn(-c2ccccc2)c(C(=O)NC[C@H](c2ccccc2)[NH+]2CCCCC2)c1', 'Cc1cncc(NC(=O)c2nnc(-c3ccc4ccccc4n3)s2)c1', 'CC[C@@H](COC)S(=O)(=O)c1ccc(OC)c(OC)c1', 'CC[C@@H](OC(=O)CN1C(=O)/C(=C/c2ccc(Oc3ccccc3)cc2C)SC1=S)C(=O)[O-]', 'CC[C@@H](NC(=O)Cc1cc(C(C)C)no1)C(=O)OC', 'CCn1c(C2CCCCC2)ncc2c(C)nn(CCC(=O)Nc3cccc(F)c3Cl)cc21', 'CCn1cc([C@@H]2C[C@@H]2C(=O)O[C@H](C)C(=O)NCC[C@@H]2CCCO2)s1', 'CCn1cnc(N[C@](C)(CO)N2CCOCC2)c1C', 'Cc1cc(C(=O)N[C@@H](C)c2ccc(Cl)cc2)c2ncnn2c1', 'Cc1cc(=O)oc2cc(NC(=O)/C=C/c3ccccc3)ccc12', 'Cc1cc(NC(=O)C(=O)N[C@@H](C)c2ccsc2)n(C)c1', 'Cc1cc(F)ccc1CCN(C(=O)[C@H](C)[N+](C)(C)C)c1ccccc1', 'Cc1cc([C@@H]2CCCCN2c2ncnc3[nH]ccc23)nc2ccccc12', 'Cc1cc(Cl)ccc1NC(=O)CSC1=N[C@@H]2CS(=O)(=O)C[C@@H]2N1c1ccccc1', 'Cc1cc(Br)ccc1NC(=O)c1ccc(F)cc1Br', 'C=C(C#N)[C@@H](C#N)C[NH+](C)C/C(C)C', 'C=C(C(=O)CN1C(=O)COc2ccc(S(=O)(C)C)cc21)c1ccccc1', 'C=C(C)Cn1c(C)c(C(C)=O)c2c1ncn2C', 'C=C(C=S)C(=O)NCC1(O)CCCCC1', 'Nc1nonc1SCc1ccccc1OCc1ccc(Cl)cc1', 'Nc1nonc1C(=O)Nc1sccc1C(F)(F)F', 'Nc1n[nH]c2c1[C@@H](c1ccccc1)CN(c1ccccc1Cl)C2', 'Nc1[nH]nc2nc(CN3C[C@H](CCO)C[C@@H]3c3ccccn3)ccc12', 'NS(=O)(=O)C[C@H]1Nc2cc(Cl)ccc2N2CCCCC[C@@H]12', 'NS(=O)(=O)C[C@H]1CCCC[NH+]1Cc1cccnc1', 'NS(=O)(=O)C[C@H]1Oc2ccccc2C(=O)N1c1ccc(F)cc1F', 'Oc1cccc([C@H](COC2CC[NH2+]CC2)N2CCOCC2)c1', 'Oc1cccc([C@H]2C[C@H]2C(=O)N2CCC(OCc3ccccc3)CC2)c1', 'Cc1sn(-c2ccc(F)cc2)c(=O)c1/C=C(/C#N)C(=O)[O-]', 'O=[N+]([O-])c1ccc[nH+]c1N1CCC(n2cncn2)CC1', 'Oc1ccc(O[C@@H](C(=O)[O-])c2ccccc2)cc1', 'Oc1ccc(O[C@@H]2CCN(C(=O)Cc3c[nH]c4ccc(Cl)cc34)C2)cc1', 'Oc1ccc(O[C@H](Cc2cscn2)C(=O)c2cccc3ccccc23)cn1', 'Oc1ccc(O[C@H]2CCOC3(CCOC3)C2)cn1', 'Cc1snnc1CN(C)C(=O)c1ccc([S@](C)=O)cc1', 'Cc1cnc([C@@H]2[C@@H](c3ccccc3)OC[C@@H]2NC(=O)/C=C/c2ccccc2)s1', 'Cc1cncnc1N/N=C1\\C(=N)N(C)C2=c3ccccc3=[NH+][C@H]21', 'Cc1cncc(C(C)C)c1NC(=O)[C@@H](C)Oc1ccccc1C#N', 'Cc1cnc2c(NCc3ccccc3)nnn2c1', 'O=[N+]([O-])c1ccnc2cc(C(F)(F)F)cc(-c3cc(Cl)cnc3C3CC3)c12', 'O=[N+]([O-])c1ccnc2c1CN(C(=O)CCCc1cccs1)C[C@@H]2O', 'O=[N+]([O-])c1c(F)cccc1NCC(=O)N1CCc2ccccc2C1', 'O=[N+]([O-])c1c(F)cc([O-])cn1', 'OCc1n[nH]c(=O)c2cc(S(=O)(=O)N3CCCC3)ccc12', 'OCc1n[nH]c(Nc2cccc3ccccc23)n1', 'OCc1n[nH]cc1CNCc1ccc([N+](=O)[O-])cc1', 'OCc1n[nH]c(COc2ccccc2OCc2cccc(Cl)c2)n1', 'OCc1n[nH]c(=S)n1/N=C/c1ccc2[nH]cc(-c3ccc(Cl)cc3)c2c1', 'OCc1n[nH]c(O)c1-c1ccccc1', 'OCc1n[nH]c(Nc2ccc(F)cc2)c1-c1ccccc1', 'OCc1n[nH]c([C@@H](C#N)c2ccc(Cl)cc2)c1Cl', 'OCc1n[nH]c(-c2ccc(N3CCCC3)c[nH+]2)c1[N+](=O)[O-]', 'OCc1n[nH]c(Sc2nc(-c3ccccc3)no2)c1[N+](=O)[O-]', 'OC[C@@]1(NCC2CC2)CCC[C@@H]1CCOc1cccnc1', 'OC[C@@]1(Nc2cccc([N+](=O)[O-])n2)CCCc2ccccc21', 'OC[C@@]1(N[C@@H]2CCS(=O)(=O)C2)CCCC1', 'OC[C@@]1(N2CCc3ccccc3C2)CCC[NH+]1C1CCCCC1', 'O=[N+]([O-])c1cc(Nc2cc([N+](=O)[O-])c(Cl)cc2)ccc1Br', 'O=[N+]([O-])c1cc(N2CCOCC2)cs1', 'O=[N+]([O-])c1cc(N[C@@H]2CCN(C(=O)c3ccccc3F)C2)ccc1F', 'O=[N+]([O-])c1cc(N[C@H]2CCc3c(Br)cccc32)ncn1', 'O=[N+]([O-])c1cccc2[nH]ccc12', 'O=[N+]([O-])c1cccc2c1NC(=O)C[C@H]2c1ccccc1NCC1(Cl)CCCC1', 'O=[N+]([O-])c1cccc2ncccc12', 'O=[N+]([O-])c1ccn(CC[C@@H](O)COc2ccccc2)n1', 'O=[N+]([O-])c1ccn(C(=O)CBr)c1', 'O=[N+]([O-])c1ccn(Cc2ccccc2)c1', 'O=[N+]([O-])c1ccn(CN2CC[C@@H](c3ccccc3)C2)c1', 'O=[N+]([O-])c1ccn(C[C@@H](Cl)Oc2cccc(F)c2)c1', 'O=[N+]([O-])c1ccn(Cn2nnnc2-c2ccccc2)c1', 'O=[N+]([O-])c1ccn(C[C@H](O)c2cccs2)c1', 'O=[N+]([O-])c1ccc[nH+]c1N1CCC(NC2CCOCC2)CC1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1-c1ccccc1', 'O=[N+]([O-])c1c(Cl)ncnc1N[C@@H](c1ccccc1)c1ccccc1', 'O=[N+]([O-])c1c(Cl)n2ncnc2n1Cc1ccccc1', 'O=[N+]([O-])c1c(Cl)nnc2c1CN(Cc1ccccc1Cl)[C@H]2NC(=O)c1ccccn1', 'O=[N+]([O-])c1c(Cl)nsc1Cl', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(Br)cc1N1CCN(c2ncccn2)CC1', 'O=[N+]([O-])c1c([N+](=O)[O-])c(Cl)ccc1NC1CC1', 'O=[N+]([O-])c1c(-c2cccnc2)cnnc1O[C@@H](CN1CCOCC1)c1cccs1', 'O=[N+]([O-])c1c(F)cc(C[S@@](=O)/C=C(\\Cl)Cc2ccccc2)c(Cl)c1O', 'O=[N+]([O-])c1c(F)cccc1NC[C@@H]1CCC[NH2+][C@@H]1CO', 'O=[N+]([O-])c1c(F)cc2c(C(F)(F)F)ncnc2c1', 'Oc1cccc(SCc2nc(-c3ccccc3)no2)c1', 'Oc1cccc(S[C@@H]2C[C@@H](O)[C@@H](O)[C@@H]2O)c1', 'Oc1cccc2ncc(C[NH2+][C@@H]3CCSc4ccccc43)n12', 'Oc1cccc2ncccc12', 'Oc1cccc2nccnc12', 'Oc1ccc(/N=c2/sccc(=O)n(Cc3ccc4c(c3)OCO4)c2=O)cc1', 'O=C(/C=C\\C1C2CC(CC(C2)C1)[C@@H](CO)c1ccccc1O)N1CCc2ccccc21', 'O=C(/C=C\\Nc1nccs1)NCCc1ccccc1', 'O=C(/C=C\\c1ccc([S@](=O)C(F)F)cc1)NCCNC(=O)c1ccccc1', 'COc1nc2nc(N(C[C@@H]3CCCO3)C(C)C)c(C)nc1-2', 'Clc1ccsc1-c1nnc(CSc2ncccn2)[nH]1', 'Clc1ccsc1F\n', 'Clc1ccsc1[C@H](N[C@@H]1CCSC1)c1ccccc1', 'Clc1ccsc1[C@H]1[NH+]=c2ccc(O)cc2=[NH+]C1=O', 'O=C(c1cnc([C@H]2CCC[NH+](C/C=C\\c3cccs3)C2)o1)N1CCCC1', 'O=C(c1cnccn1)N(Cc1cccc(Cl)c1)c1nc2ccc(F)cc2s1', 'OC[C@H]1CN(Cc2ccccc2CO)C[C@@H]1c1ccco1', 'OC[C@H]1O[C@@H](CO)[C@H](O)[C@H](O)[C@@H]2CCCC[C@H]12', 'OC[C@H]1[C@H](Cc2ccc(O)cc2)NC(=O)N1Cc1ccc(F)cc1', 'N#Cc1nsnc1CCC(=O)NC1CCc2cccc(F)c21', 'N#Cc1n[nH]c(=O)c2cc(Cl)cc(-c3ccc(F)cc3F)c12', 'N#Cc1nc(-c2ccsc2)nc2c1CC[C@@H](c1ccccc1)C2', 'Oc1cc(O)c([N+](=O)[O-])cc1OCCCC(F)(F)F', 'Oc1cc(O)c(Oc2ccc(N3CCCC3=O)cc2)nc1NC[C@@H]1CCCO1', 'Oc1cc(O)c(N[C@@H]2CC[C@H]([NH3+])C2)cc1Cl', 'Oc1cc(O)c(-c2ccccc2)nc1OC[C@H]1CCCO1', 'Oc1cc(O)c(Cl)cc1/C=C/C(=O)[O-]', 'Oc1cc(O)c(Br)cc1OC[C@@H]1COc2ccccc2O1', 'Oc1cc(O)c(/C=C/C(=O)[O-])cc1Br', 'N=Cc1cc[nH+]c(N2CCCC2)c1N[C@H]1CCO[C@]2(CO2)[C@H]1C', 'N=Cc1cc([N+](=O)[O-])c(Cl)cc1Oc1ccccc1Cl', 'N=Cc1cccc2c1OCCN(C(=O)/C=C\\c1nc(C)co1)C2', 'N=Cc1cc2cc(OC)ccc2c(Cl)c1Cl', 'ClC[C@@H]1OCCC[C@@H]1[NH2+]C1CC1', 'ClC[C@@H]1OCC(CN2CCC[C@@H]2C(N)=O)C1', 'ClC[C@@H]1OCCc2c1[nH]c1ccccc21', 'ClC[C@@H]1OCCOc2ccccc21', 'ClC[C@@H]1OCC[C@@H]1CN[C@H]1CCCC[C@@H]1CO', 'ClC[C@@H]1OCC[C@H](Sc2cccc(Cl)c2)C1', 'ClC[C@@H]1OCC[NH+]1[C@H]1CCOC2(CCSCC2)C1', 'O=[N+]([O-])c1cc(Cl)ccc1Oc1ncccn1', 'O=[N+]([O-])c1cc(C(F)(F)F)ccc1Oc1ccc(Oc2ccccc2)[nH]c1=O', 'O=[N+]([O-])c1cc(C[NH+]2CCC[C@@H](CN3CCOCC3)N(c3ccccc3)C2)ccn1', 'O=[N+]([O-])c1cc(C2CCCCC2)n2ncc(Cl)c2c1', 'O=C([C@H](Cl)c1ccccc1)c1cc(Br)ccc1F', 'O=C([C@H](Cl)Cc1ccccc1)c1ccccc1', 'O=C([C@H](Cl)c1ccccc1)c1ccc([N+](=O)[O-])cc1F', 'O=C([C@H](Cl)Br)c1ccccc1', 'O=C([C@H](SCc1cccs1)N1C(=O)CSC1=O)Nc1ccc(Br)cc1', 'O=C([C@H](SC1CCCC1)c1ccccc1)Nc1ccc(Oc2ccccc2)cc1', 'O=C([C@H](SCO)c1ccc(Br)cc1)N1CCN(Cc2ccccc2)CC1', 'O=C([C@H](SC[C@@H]1CCCCO1)c1c[nH]c2ccccc12)C(=O)[O-]', 'O=C(c1c[nH]nc1-c1ccc(F)cc1)c1cc(S(=O)(=O)N2CCCCC2)ccc1O', 'O=C(c1c[nH]nc1C1CC1)c1cc(-c2ccc(Br)cc2)nc2ncnn12', 'O=C(c1c[nH]nc1N1CCOCC1)N1CCN(C[C@H]2CCCC[NH+]2Cn2cncn2)CC1', 'O=C(/N=C/C1=C(N)NC(=O)N(c2ccc(Cl)cc2)c2ccccc21)c1ccccc1', 'O=C(/N=C/N=C\\c1cccc(O)c1)[C@H]1CCCN(S(=O)(=O)Cc2ccccc2)C1', 'O=C(/N=C/c1ccc(F)cc1)c1ccc(NC(=O)c2cccc([N+](=O)[O-])c2)cc1', 'Oc1ccccc1[C@@H](Cc1ccc(Br)s1)N1C[C@H]2CCCC[C@@H]2C1', 'Oc1ccccc1[C@@H]1CCCN1c1nc2ccccc2[nH]1', 'Oc1ccccc1OC[C@H](O)c1ccc(Cl)c(Cl)c1', 'Oc1ccccc1OS(=O)(=O)c1ccccc1', 'Oc1ccccc1Oc1ccc(N2CCOCC2)cc1Cl', 'Oc1ccccc1O[C@@H](c1ccccc1)c1c(N)cnn1C', 'Oc1cccc(F)c1NC(=O)c1c[nH]c2ccccc12', 'Oc1cccc(Br)c1Oc1ccccc1C(F)(F)F', 'Oc1cccc(F)n1', 'O[C@@H](S[C@@H](Cc1ccccc1)c1ccc(Cl)c(Cl)c1)c1cc(Cl)ccn1', 'O[C@@H](S[C@@H]1CCOC2(CCOCC2)C1)C(F)(F)F', 'Oc1ccc([N+](=O)c2cc(Br)ccc2F)cc1O', 'Oc1ccc([N+](=O)[O-])c(/C(=N/c2ccc(Cl)cc2)[n+]2ccccc2Cl)c1', 'Oc1ccc(/C=N/c2cccc(Br)c2)cc1', 'Oc1ccc(/C=N/O[C@H]2CC(=O)N(c3ccccc3)C2)cc1Cl', 'Oc1ccc(/C=N/Nc2ccc(Cl)cc2Cl)c(Br)c1', 'Oc1ccc(/C=N/[C@@H]2CCCOC2)cc1', 'Oc1ccc(/C=N/Sc2nnc(N/N=C\\c3ccc(Br)cc3)o2)cc1', 'Oc1ccc(/C=N/[NH+]C(=O)COc2ccccc2Br)cc1Br', 'COc1nc(-c2ccccc2F)cc(S(N)=[NH+]/CCNC(=O)c2ccc3c(c2)OCO3)c1Cl', 'COc1nc(-n2nc(C)cc2CC(=O)N2CCCc3cc(C)ccc32)cs1', 'O=[N+]([O-])c1cccc2ncn(Cc3ccccc3)c12', 'O=[N+]([O-])c1cccc2nnc(C[NH+]3CCC[C@@H](CCc4cccnc4)C3)nc12', 'OCc1nn(-c2ccccc2)c(=S)n1C1CC1', 'OCc1nn(Cc2ccco2)c2[nH]ccc12', 'OCc1nn(S(=O)(=O)c2ccccc2)c2ccccc12', 'OC[C@@H](NCc1cc(Br)cs1)c1ccc(Cl)cc1', 'OC[C@@H](Nc1ncccc1C(F)(F)F)N1CC[C@@H](C)[C@@H](O)C1', 'OC[C@@H](N[C@@H]1CCN(CC2CC2)C1)c1nc2ccccc2[nH]1', 'OC[C@@H](NS(=O)(=O)c1ccc(Cl)cc1)c1cccc(F)c1', 'OC[C@@H](N[C@H]1CCSc2ccccc21)[C@@H]1CCC(=O)N1', 'O=C([C@H](c1ccccc1)N1CCC(OC2CCCCC2)C1)N1CCN(c2ccccc2)CC1', 'O=C([C@H](c1c(Cl)cccc1Cl)c1ccc(Cl)cc1)c1ccccc1[N+](=O)[O-]', 'O=C([C@H](c1ccccc1)n1cccc1)c1cccc(Cl)c1', 'O=C([C@H](Cl)CCSc1ccccc1)NC[C@@H]1CCCO1', 'O=C([C@H](Cl)C(F)(F)F)N1CCOCC1', 'O=C([C@H](Cl)Cc1ccccn1)[C@@H]1CCC[C@@H](O)C1', 'O=C([C@H](Cl)C1CC1)N1c2ccccc2C[C@@H]1CCc1ccc(C(F)(F)F)cc1', 'O=C([C@H](Cl)CS)c1cccc(F)c1', 'O=C([C@H](Cl)CBr)c1ccccc1', 'O=[N+]([O-])c1ccncc1COc1cccc2c1C(=O)CCC2', 'O=[N+]([O-])c1ccncc1OCCC(F)(F)F', 'O=[N+]([O-])c1ccncc1N1CCSc2ccccc21', 'O=[N+]([O-])c1ccncc1F', 'O=[N+]([O-])c1ccncc1[C@@H]1CCCN(c2nccc(-c3ccc(F)cc3)n2)C1', 'O=[N+]([O-])c1ccncc1-c1ccc([N+](=O)[O-])cc1', 'O=[N+]([O-])c1ccncc1Sc1ncc[nH]1', 'O=[N+]([O-])c1ccc[nH+]c1C[NH2+][C@H]1C(CO)=C(C)CC(=O)N1c1ccc(Br)s1', 'O=[N+]([O-])c1ccc[nH+]c1Nc1ccc(/C=C/C(=O)[O-])cc1', 'O=[N+]([O-])c1ccc[nH+]c1I', 'O=[N+]([O-])c1ccc[nH+]c1[C@@H](N1CCOCC1)[C@@H]1CCS(=O)(=O)C1', 'O=[N+]([O-])c1ccc[nH+]c1-c1nc2ccc(OCC(=O)Nc3c(Cl)cccc3Br)nc2[nH]1', 'O=[N+]([O-])c1ccc[nH+]c1SCCF', 'C[C@@H](NS(=O)(=O)c1ccccc1[N+](=O)[O-])c1ccc(F)cc1F', 'C[C@@H](c1noc(-c2ccc(O)cc2)n1)N1CC[NH+](C[C@@H]2CCCO2)CC1', 'C[C@@H](c1nc[nH]n1)n1ccc(NC(=O)c2ccc(C#N)cc2)n1', 'OCc1ncc(CCOCc2ccccc2)s1', 'OCc1ncccc1CNC[C@H]1CC[NH+]=CC1', 'OCc1ncco1', 'OCc1nccnc1C[C@@H](C[C@@H]1CCOC1)NC(=O)c1cccnc1', 'OCc1nccs1', 'OCc1ncc[nH]1', 'Oc1cc(-n2nnnc2COCC2CC2)cc(OC2CCCCC2)c(C(F)(F)F)c1=O', 'Nc1nonc1COc1ccc(Br)cc1', 'Nc1nonc1NCC(=O)Nc1cc(Cl)ccc1Cl', 'Nc1nonc1-c1ccccc1', 'Nc1nonc1Sc1ccccc1', 'Nc1nonc1/N=N/c1ccc(-c2ccccc2Cl)c(Cl)c1', 'Nc1n[nH]c([C@@H]2CCCCC[C@@H]2[NH3+])n1', 'Nc1n[nH]c(-c2cccc(F)c2)c1NC(=O)c1ccc(S(=O)(=O)C(F)F)cc1', 'Nc1n[nH]c(/N=C2\\C(=O)NC(=O)C([O-])=CC2=O)n1', 'OC1=[N+]2cccc2C(=O)OC1', 'OC1=C(c2ccccc2)/C(=C/c2ccc(F)cc2)C1', 'OC1=NC(SCC(=O)Nc2cccc(Cl)c2)=NN1', 'Oc1ccc([N+](=O)c2oncc2C2CC2)cc1F', 'Oc1ccc([N+](=O)c2ccccc2)cc1Br', 'OCc1n[nH]cc1-c1cccc(Cl)c1', 'OCc1nnsc1C(=O)c1ccc(Br)s1', 'OC[C@H]1CN(c2nc(-c3ccc(Cl)c(Cl)c3)no2)CCO1', 'OC[C@H]1CC[C@]2(CCc3c(F)cc(Br)cc3O2)C1', 'OC[C@H]1C(=O)C[C@H](C(=O)N(Cc2ccc(Cl)cc2)c2ccccc2)C1', 'Oc1ccc(F)cc1CN1CCN(c2[nH+]ccc3ccccc23)CC1', 'Oc1ccc(F)cc1O/N=C/c1ccc(S(=O)(=O)[N-]c2cccc(Cl)c2)cc1', 'Oc1ccc(F)cc1NC(=O)c1cc(F)ccc1O', 'Oc1ccc(F)cc1-c1nnc(SCCc2ccccc2)n1-c1ccccc1', 'Oc1ccc(F)cc1SCC(=O)Nc1ccc2c(c1)COC2', 'Oc1ccc(F)cc1[C@H]1Oc2ccccc2C(=O)N1C[C@@H](O)C1CC1', 'Oc1ccc(O[C@@H]2CCC[NH2+]C2)cc1', 'Oc1cc(-n2ncc(Cl)c2Cl)ccn1', 'Oc1cc(-n2nccc2C(=O)c2ccccc2OCc2ccc(Cl)cc2Cl)cc2c1NCCC2', 'Oc1cc(F)nc(Nc2ccc(C(F)(F)F)cc2)c1', 'Oc1cc(F)ncc1CNc1cc(C(F)(F)F)c2c(c1)CCCC2', 'Oc1cc(F)nc(CN2CC[C@@H](N3CCO[C@@H](CO)C3)C2)c1', 'Oc1cc(F)nc([S-])c1SCc1ccccc1Cl', 'Oc1cc(F)nc(NC[C@H]2CCCO2)c1', 'Oc1cc(F)nc([C@@H]2CCCN2c2ncccn2)c1', 'Oc1cc(F)nc(-n2cnnn2)c1Cl', 'Oc1cc(F)nc(Sc2ccccc2Br)c1', 'Oc1cc(F)nc(Cl)n1', 'OCc1n[nH]cc1C[NH+]1CCC[C@H](Oc2cccc(Cl)c2)C1', 'OCc1n[nH]cc1Oc1ccc(C(=O)Nc2ccc(Cl)cc2)cc1', 'OCc1n[nH]cc1[C@@H]1CCC[NH2+]C1', 'OCc1n[nH]cc1[C@H](CCO)c1ccc(Cl)cc1', 'O=C([O-])CC[C@@H]1COc2ccccc21', 'O=C([O-])C(C1CCOCC1)C1CCCC1', 'O=C([O-])Cc1ccc(NS(=O)(=O)c2ccccc2F)cc1', 'O=C([O-])COc1nc(CCc2ccsc2)n[nH]1', 'O=C([O-])CN1N=N[C@@H](c2ccc([N+](=O)[O-])cc2)SC1=S', 'O=C([O-])C[C@@H](N[C@@H]1CCc2c(Br)cccc21)C(=O)[O-]', 'O=C([O-])CSc1ccccc1C(=O)Nc1ccc(OC2CCCC2)cc1', 'O=C([O-])C[C@H]1CCC[NH+](C[C@@H](O)c2cc(Cl)ccc2F)C1', 'Cc1nonc1CCCC(=O)NNC(=O)C[C@@H](O)c1ccccc1C', 'Cc1sn(-c2ccccc2)c1C(=O)N[C@H](c1cccc(Cl)c1)c1ccccc1', 'Clc1ccccc1COCc1ccc(OC[C@H]2CCCO2)cc1Cl', 'Clc1ccccc1O/C(=C/c1ccc(Cl)c(Cl)c1)N1CCCCC1', 'Clc1ccccc1NC1CC[NH+](C2CCCC2)CC1', 'Clc1ccccc1-n1nnnc1SCc1cc(Cl)c2c(c1)OCO2', 'Clc1ccccc1[C@H]1C[C@@H](CC(=O)Nc2cccc(C(F)(F)F)c2)N(Cc2ccc(F)cc2)C1', 'Clc1cccc2[nH]c(=O)n([C@@H](CN3CCO[C@H](O)CCCO)C3)c12', 'Clc1cccc2c1CCC[C@H]2[NH2+]Cc1ccc(OC(F)F)cc1', 'Clc1cccc2nsnc12', 'Clc1ccc(Br)cc1/N=C/C(=CC1=c([O-])c([O-])c(Cl)c1=O)N1CCCCCC1', 'Clc1cc(F)c([N+](=O)[O-])cc1OC[C@@H]1CCCO1', 'Clc1cc(F)c(C[S@](=O)CS(=O)(=O)N2CCCC[C@@H]2CO)cc1', 'Clc1cc(F)c(OCc2ccccc2F)cc1C1CC1', 'Clc1cc(F)c(I)s1', 'Clc1cc(F)c(NCCCC[NH2+]C2CC2)cc1', 'Clc1cc(F)c(F)cc1[C@H](O)c1cccc(Br)c1', 'Clc1cc(F)c([C@@H](CF)N[C@@H]2CCOC3(CCOCC3)N2)cc1', 'Clc1cc(F)c(Cl)cc1NCCc1nc(Cl)ns1', 'Clc1cc(F)c([O-])c(C[NH+]2C[C@@H](CO)C2)c1', 'Clc1cc(F)c([C@H](Cn2cc(Br)cn2)N(C2CCCC2)S(C)(=O)=O)cc1', 'Clc1cc(F)c(/C=N\\NC(=O)[C@H]2CSCCS2)cc1Cl', 'N[C@@H](NC(=O)c1ccccc1OCc1ccco1)C1CCC1', 'N[C@@H](Nc1ccc(F)c(Br)c1)c1ccccc1', 'N[C@@H](N1CCN(c2ccc(F)cc2)CC1)c1cc(F)cc(Br)c1', 'N[C@@H](S/C=C/c1ccc(Cl)cc1)c1nccn1C', 'N[C@@H](S/N=C(\\C)c1ccco1)N1CC=C(Cc2ccccc2)C1=O', 'N[C@@H]1[NH2+]CC(C)(C)[C@H]1C[C@@H](CO)c1ccccc1', 'N[C@@H]1[NH2+]N2CC(c3ccccc3)=N[C@H]2C1', 'Oc1cccc(Br)c1S(=O)(=O)NCc1cccs1', 'Oc1cccc(-c2csc(CN(c3ccc(Cl)cc3)S(=O)(=O)c3ccccc3)n2)c1', 'Oc1cccc(F)n1', 'Oc1cccc(Br)c1Oc1ccc(Br)cc1', 'Oc1cccc(Br)c1Cn1cccc(Br)c1=O', 'Oc1cccc(-n2cccc2C[NH+]2CCC(N3CCCC3)CC2)c1', 'Oc1cccc(CCNc2c(Cl)cc(Cl)cn2)c1', 'Oc1cccc(C(=O)Nc2ccccc2-c2ccco2)c1', 'Oc1cccc(C2=NN([C@H]3[C@@H](C(=O)[O-])C[C@H](O)N(c4ccccc4)[C@H]3c3ccccc3)C2)c1', 'Oc1cccc(CN(CCN2CCOCC2)C(=S)N2CCCCCC2)c1', 'Oc1cccc(CCl)c1', 'Oc1cccc(C[NH+]2CCC[C@H](Nc3ccc(Cl)cc3)C2)c1', 'Oc1cccc(C[NH2+]Cc2ccc(Oc3ccccc3)cc2)c1', 'O[C@@H]1c2ccccc2CN1Cc1ccc(-c2ccccc2)cc1', 'O[C@@H]1c2ccoc2C(=O)N1[C@H]1CCC[C@@H]1CC(=O)Nc1ccc(F)cc1', 'O[C@@H]1c2cc(F)ccc2[C@H]2CCCC[C@@H]12', 'O[C@@H]1c2ccccc2CC[C@@H]1O', 'O[C@@H]1c2ccc(Br)cc2C[C@H]1CC(=O)Nc1ccc(OCC=O)cc1', 'O[C@@H]1c2ccc3ccccc3c2CN1C(=O)CS[C@H]1CCS(=O)(=O)C1', 'O[C@@H]1c2ccccc2-c2nccnc2S1(=O)=O', 'C=CC#CCN(C)C(=O)C1CCN(C(=O)c2cccc(F)c2)CC1', 'C=C[C@@H]1Cn2c(nnc2S[C@H](C)C(=O)N(CC)Cc2ccccc2)C1', 'C=C[C@@H]1N[C@@](CN2C(=O)C[C@@H](NCc3ccc[nH]3)N2)c2ccccc21', 'C=C[C@@H]1[C@@H](C)CC[C@@]1(O)C[NH+]1CCCC1', 'Oc1cc(-n2cc[nH+]c2C2CC2)cc(N2CC(=O)Nc3ccc([N+](=O)[O-])cc32)c1', 'Oc1cc(-n2cc(/C=C3/C(=N/c4ccccc4)c4ccccc4S3)C2=O)ccc1O', 'Oc1ccc([N+](=O)[O-])cc1Cl', 'Oc1ccc([N+](=O)[O-])c(/C=C2\\S[C@@H]3CS(=O)(=O)C[C@@H]3N2C(=O)c2ccco2)c1', 'Oc1ccc(/N=c2/nc(-c3ccccc3)[nH]n3ncnc23)cc1O', 'Oc1ccc(/N=c2/sc3c(s2)CCN3C(F)(F)F)c(F)c1', 'Oc1ccc(/N=c2/[nH]cc[nH]2)nc1Cl', 'CC(C)C[C@@H](C[NH3+])C(=O)Nc1ccncc1', 'CC[S@](=O)[C@@H]1CCCC[C@@H]1NS(=O)(=O)N1CCOCC1', 'O=C(c1ccsc1I)N(CC1CC1)C1CC1', 'O=C(c1ccsc1)N1CCC(Oc2ccc(F)cc2)CC1', 'O=C(c1ccsc1C#N)[C@@H]1Cc2ccccc2C(=O)N1c1ccc(F)cc1', 'O=C(c1ccsc1)N1CCN(S(=O)(=O)OC[C@@H]2CCCO2)CC1', 'O=C(c1cn(-c2ccccc2)nc1-c1cc2ccccc2nc1N1CCOCC1)c1ccc2ccccc2c1', 'O=C(c1cn(Cc2ccco2)nc1-c1ccco1)c1ccc(Br)cc1', 'O=C(c1cn2ccsc2n1)N1CC[C@](N2CCOCC2)C1', 'O=C(c1cn(-c2ccccc2)nc1-c1cnn(-c2ccccc2)c1)C1CCOCC1', 'O=C(c1cn(Cc2cccs2)nc1-c1ccc(Cl)cc1)N1CCOCC1', 'O=C(c1cn(C2CC[NH2+]CC2)nn1)N1CCN(c2ccc(F)cc2)CC1', 'O=C(c1cn(C[C@@H]2CCCO2)cn1)c1ccccc1', 'O=C(c1cn(C[C@H]2CCCO2)cc1)N1CCC[C@@H]1CF', 'O=C(CNc1ccccc1[N+](=O)[O-])c1ccccc1', 'O=C(CS(=O)(=O)C[C@H]1Sc2ccccc2NC1=O)NC[C@@H](O)c1ccccc1', 'O=C(c1c[nH]cn1)c1cccc(CN2CCC[C@H](O)C2)c1', 'O=C(COc1ccccc1F)NCC[NH+]1CCc2ccccc21', 'O=C(c1ccsc1)N1CCN(Cc2ccccc2)CC1', 'O=C(c1ccsc1)c1ccc(COCCO)cc1', 'O=C(c1ccsc1)[C@@H]1CC(c2ccc(Cl)cc2)=NO1', 'O=C(/C(c1ccccc1)[C@@H]1CC=CCC1)c1ccc([N+](=O)[O-])cc1', 'O=C(c1c[nH]c2ccccc12)N1CCC[C@H](n2ccnc2)C1', 'O=C([C@H]1COCCN1S(=O)(=O)c1ccccc1Cl)N1CCC(c2nccs2)CC1', 'O=C([C@H]1CO[C@H]1c1ccccc1)N1CCCN(c2nc3ccccn3[nH+]2)CC1', 'O=C([C@H]1COc2cccc3cccc(c23)CN1C(=O)n1ncc(Br)c1Cl)N1CCCC1', 'O[C@@H]([C@@H](CCc1ccsc1)NCCCc1ccc2c(c1)OCCO2)C1CC1', 'O[C@@H]([C@@H](c1ccc(F)cc1)[C@@H]1CCOC1)N1CCOCC1', 'O[C@@H]([C@@H](O)c1ccccc1)CN1CCCCC1=O', 'O[C@@H]([C@@H](N[C@@H](CO)c1ccccc1)c1ccccc1)c1ccccc1', 'O[C@@H]([C@@H](n1cc(C(F)(F)F)c2ccccc21)C1CCCC1)N1CCOCC1', 'O[C@@H]([C@@H](SCC(F)(F)F)C(F)(F)F)NC[C@H]1CCCO1', 'O[C@@H]([C@@H](Cl)CO)C(=O)OCc1ccccn1', 'O[C@@H]([C@@H]([NH3+])CO)C(F)F', 'O[C@@H]([C@@H]([NH2+]Cc1ccccc1)Sc1ccccc1)c1cccc(F)c1', 'Clc1nnns1\n\n', 'Nc1[nH]nc2c1CCCCC[NH+](Cc1ccccn1)C2', 'C[C@@H](NC(=S)Nc1nccs1)C(=O)NC1CC1', 'C[C@@H](NCc1ccc(Cl)cc1Cl)c1ccc(Cl)cc1', 'C[C@@H](Oc1ccccc1[N+](=O)[O-])c1nc(C(F)(F)F)ccc1Cl', 'C[C@@H](N[C@@H](CO)C(C)C)c1ccccc1', 'C[C@@H](N[C@@H]1CC(C)(C)COC1)C(=O)N[C@@H]1CCN(c2c(F)cccc2F)C1', 'C[C@@H](Oc1ccccc1F)C(=O)Nc1cnn(CC(=O)NC2CC2)c1', 'C[C@@H](Oc1ccc(F)cc1)C(=O)NC1CC[NH+]([C@@H](C)c2ccccc2)CC1', 'C[C@@H](NS(=O)(=O)c1ccc2ccccc2c1F)C(=O)NCCCc1ccccc1', 'C[C@@H]([C@@H](O)COc1ccccc1)N(C)c1ccccc1', 'C[C@@H](Nc1cccc(C#CN2CCOCC2)c1)c1nnc(-c2ccccc2)n1C', 'C[C@@H](Oc1ccc(Cl)cc1Cl)C(=O)Nc1ccc(C(=O)O[C@H](C)C(F)(F)F)cc1', 'C[C@@H](Oc1ccncc1)C(=O)Nc1nc(-c2ccccc2)ccc1F', 'C[C@@H]([C@@H](O)CCO)N(C)C(=O)c1ccc(Br)cc1F', 'C[C@@H]([C@@H](O)c1ccccc1)N(C)CC(=O)Nc1ccc(F)cc1Cl', 'C[C@@H]([C@@H](O)[C@@H](c1ccncc1)n1cccn1)N1CC[NH+](C)CC1', 'C[C@@H]([C@@H](O)[C@@](C)(C(N)=O)[NH2+]C1CC1)c1ccccc1', 'COc1ncc([N+](=O)[O-])c(/C=N/NC(=O)c2ccc(C)cc2OCC)c1C', 'COc1ncc(CNS(=O)(=O)Cc2ccc(F)c(F)c2)cc1OC', 'COc1ncc(NC(=O)N2C[C@H](C)O[C@H](C)C2)c(C)c1OC', 'COc1ncc(F)c(C[NH2+]Cc2ccc(SC)cc2)c1C#N', 'COc1ncc(-c2ncc[nH]2)cc1C#N', 'COc1ncc(SCC#N)c(-c2ccc(Cl)cc2Cl)n1', 'COc1ncc(Cl)c([C@@]2(CC)COc3ccccc32)c1[N+](=O)[O-]', 'COc1ncc(Br)cc1C(=O)N[C@@H]1C[C@@H](C)N=C(C#N)[C@H]1c1ccccc1Cl', 'COc1ncc(/C=C/C(=O)Nc2ccc(Cl)nn2)c(C)c1CC', 'CNC(=O)c1ccc([C@H](C)[NH2+]C2CCN(c3ccccc3F)CC2)cc1', 'CC(C)CC[C@H](C)[NH2+]Cc1cccc(O[C@@H](C)C(=O)OC(C)(C)C)c1', 'CC(C)(C)C(=O)Nc1c(NCc2[nH+]ccn2C2CCCCC2)cnn1-c1ccc(Cl)cc1', 'CC(C)c1cc(F)ccc1S(=O)(=O)Nc1ccc(Cl)c(Cl)c1', 'CC(C)=CC(=O)C[C@@H]1C(=O)NCCN1C(=O)c1cccc(Cl)c1', 'CC(C)OCCc1n[nH]c(-c2ccccc2)n1', 'CC(C)N(Cc1[nH+]ccn1C)S(=O)(=O)c1ccc2c(c1)CCN2C(=O)OC(C)C', 'CC(C)n1nccc1NC(=O)c1cccc(Cl)c1', 'CC(C)[C@H](C(=O)Nc1ccccc1)N(C)C(=O)c1ccc2c(c1)OCCO2', 'CC1=CC[C@]2(C)CCC[C@@H](C)C[C@H]1[NH2+]CC2', 'CC1=NO[C@@H](C(=O)N2CCC[C@H]2C(=O)[O-])C1', 'OC[C@@H](c1ccco1)N1CC[C@@H](CS(=O)(=O)c2cccc(F)c2)C1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(SCc2ccccn2)c1-c1ccncc1Cl', 'O=[N+]([O-])c1c(Br)sc(CN2CCN(c3cccc(Cl)c3)CC2)c1CO', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1Nc1cccc(Cl)c1', 'O=[N+]([O-])c1c(-c2cccnc2)ncc(C(=O)Nc2ccccc2F)c1C1CC1', 'O=[N+]([O-])c1c(-c2nc3ccccc3o2)ccc2nccc(NCc3ccccc3)c12', 'O=[N+]([O-])c1c(Br)scc1C(F)(F)F', 'O=[N+]([O-])c1c(Br)sc2c1C(=O)[C@@H](c1ccccc1)[C@@H]1CC=C[C@H]21', 'O=[N+]([O-])c1c(-c2nc(C3CC3)no2)cnn1Cc1ccc(F)cc1', 'O=[N+]([O-])c1c(-c2noc(-c3ccc(F)cc3)n2)nnn1-c1ccccc1', 'O=[N+]([O-])c1c(-c2nnnn2-c2ccccc2)nc(-c2ccccc2)c2ccccc12', 'O=[N+]([O-])c1c(-c2nsc(-c3ccsc3)n2)nc2ccccc2c1[O-]', 'O=[N+]([O-])c1c(-c2n[nH]c([O-])n2)ccc2nsnc12', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1C(=O)Nc1csc2ccccc12', 'O=[N+]([O-])c1c(Cl)nc(Cl)cc1Oc1cccc(Cl)c1Cc1ccccc1', 'O=[N+]([O-])c1c(Cl)nc2ccccn12', 'O=[N+]([O-])c1c(Cl)ncc2ccc(Cl)cc12', 'O=[N+]([O-])c1c(Cl)ncnc1CS[C@@H]1CCc2ccccc21', 'O=[N+]([O-])c1c([O-])[nH]c(=O)c2ccc(F)cc12', 'O=[N+]([O-])c1c([N+](=O)[O-])c(F)cc(C(=O)NCCc2ccccc2)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1Oc1ccc(F)cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1-c1ccccc1', 'O=[N+]([O-])c1c([O-])cnn1-c1cccc2c1CCN(Cn1cncc1Cl)C2', 'O=[N+]([O-])c1c([O-])c2ncn(CCCc3ccc(F)cc3)c2nc1N', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(O)c(CN2CCCCC2)c1Cl', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(C(F)(F)F)c1C[NH2+]Cc1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccn1Cc1ccc(Cl)c2ccccc12', 'O=[N+]([O-])c1c(Cl)n2nccc2n1Cc1ccccc1Br', 'O=[N+]([O-])c1c(-c2c[nH]c3ccccc23)nc2ccccn12', 'O=[N+]([O-])c1c(-c2ccoc2)cccc1[N+](=O)[O-]', 'O=[N+]([O-])c1c([O-])ncnc1[C@@H]1CCOC1', 'O=[N+]([O-])c1c([O-])n2ncc(C3CC3)c2ncn1-c1ccc(F)cc1F', 'O=[N+]([O-])c1c([O-])nnnc1Sc1ccccc1', 'O=[N+]([O-])c1c([O-])n[nH]c1NCc1cc2ccccc2o1', 'OC1(S(=O)(=O)c2ccc3c(c2)C=CS[C@@H]3c2ccccc2)CCOCC1', 'OC1(Sc2ccccc2OCC2CCCCC2)CCCCCCC1', 'OC1(SSCc2cccc3cc(F)ccc23)CCCCC1', 'Oc1cc(Cn2c(=O)oc(C(=O)NCc3ccccc3)c2C2CC2)ccc1[O-]', 'Oc1cc(C(F)(F)F)ccc1[C@H]1CCCN(c2cc(-c3ccccc3)ncn2)C1', 'Oc1cc(Cn2ccc3cc(Cl)ccc32)c([N+](=O)[O-])ccc1=O', 'Oc1cc(Cn2ncc3cc(Cl)ccc32)ccc1O', 'O=C(c1cc([N+](=O)[O-])ccc1N1CCCCC1)N1CCC[C@H](c2nccs2)C1', 'O=C(CNC(=O)c1ccccc1)N/N=C/c1ccc2ccccc2c1', 'O=C(CNCc1cccnc1)Nc1cc(Cl)ccc1[N+](=O)[O-]', 'O=C(Cc1noc([O-])n1)N[C@H]1c2ccccc2C[C@@H]1O', 'O=C(Cc1n[nH]c2c(Cl)cccc12)Nc1ccccc1F', 'O=C(Cc1nc2ccccc2n1Cc1ccccc1)Nc1ccc2c(c1)OCCCO2', 'O=C(Cc1nnc(-c2ccc(Br)cc2)o1)Nc1ccccc1F', 'O=[N+]([O-])c1c([N+](=O)[O-])c(F)cc(Br)c1[O-]', 'O=[N+]([O-])c1c([N+](=O)[O-])c(-c2ccccc2)nc2sccc12', 'O=[N+]([O-])c1c([N+](=O)[O-])c(Br)sc1N[C@@H]1C=CS(=O)(=O)C1', 'Oc1ccccc1/C=C/C[NH+](Cc1ccc(Br)cc1C(F)(F)F)Sc1ncccn1', 'Oc1ccccc1/C(=N\\Nc1cccc(Cl)c1[N+](=O)[O-])C1(c2ccc(Br)cc2)CCCC1', 'Oc1cc(-n2nc([C@@H](OC[C@@H]3CCCO3)C2=O)c2ccccc2)ccc1F', 'Oc1cc(-n2nc(Cl)cc2SCC(F)F)nc2ccccc12', 'Oc1cc(-n2nc([O-])[nH]c2=O)c(=O)n(Cc2cccc(Cl)c2)c1', 'Clc1nnn(-c2nc(C3CC3)ns2)c1OCc1ccccc1', 'Clc1nnn(-c2ncccn2)c1Br', 'Clc1nnn(-c2ncnc(-c3ccncc3)n2)c1Cl', 'Clc1nnnn1-c1ccccc1', 'Clc1nnnn1-c1cc(F)cc(F)c1F', 'O=C(/C(c1ccnc(Cl)c1)N[C@H]1CCN(c2ccccc2)C1=O)c1cccnc1', 'O=C(c1c[nH]nc1-c1ccc(Cl)cc1)N(CC1CC1)S(=O)(=O)c1ccc(Cl)cc1', 'O=C(c1cn2ccsc2n1)N1CCC[C@@H]1c1cccc(Cl)c1', 'O=C(c1ccsc1CC1CC1)N1CCCc2ccccc21', 'O=C(c1ccsc1C(F)F)N(CC(=O)N1CCCCCC1)c1ccccc1', 'O=C(c1ccsc1C1CC1)N1CCN(c2cnn(-c3ccccc3)c(=O)c2NCc2ccccc2F)CC1', 'O=C(c1ccsc1COc1ccccc1)N1CCN(C(=O)COc2ccccc2)CC1', 'O=C(c1ccsc1CN1C(=O)CCCC1=O)Nc1cc(C(F)(F)F)ccc1F', 'O=C(c1ccsc1Cn1cnnn1)NC(c1ccccc1)c1ccccc1', 'O=C(c1ccsc1C#N)N1CCN([C@@H]2C=CC[C@@H]2C(=O)c2ccccc2)CC1', 'O=C(c1ccsc1C[NH+]1CCCCC1)c1ccc(Br)cc1', 'Oc1cccc2nnc(S(=O)(=O)NC[C@]3(O)[C@@H]4C=CC[C@@H]4C3)n12', 'Oc1cccc2nnc3cc([C@@H](CCO)c4ccccn4)nc([O-])c3c12', 'Oc1cccc2nnccc1c1ccccc12', 'Clc1nnc(NC(=O)c2ccc3c(c2)OCO3)[nH]1', 'Clc1nnc(N2CC=C(n3cc(-c4ccccc4)s3)CC2)o1', 'Clc1nnc(N[C@@H]2C[C@H]2[N+]2=NCCS2)nc1NCc1ccccc1', 'Clc1nnc(N[C@H]2CCOC3(CCOCC3)O2)cn1', 'CNc1n[nH]c(=S)n1CC(=O)N1CCO[C@@H](c2ccccc2)C1', 'CNc1nc(C(=O)N2CCC[C@H](CNc3ccc(OC)cc3)C2)c2ccccc2[nH+]1', 'CNc1nnc(SCC(=O)Nc2cccc(Br)c2)nc1N', 'CNC(=O)CCC(=O)NC[C@@H]1CCCC[C@@H]1C', 'CNC(=O)c1cc(S(=O)(=O)N2CCC[C@@H]2C2CCCCC2)cc2c1OCCO2', 'CNC(=O)N1C[C@@H](c2ccccc2F)OC1=O', 'CNC(=O)[C@@H]1CCCCN1c1ccc(NC(=O)[C@H](C(C)C)[NH+]2CCCCC2)cc1', 'CNC(=O)[C@H]1CN[C@@H]2CCCN2C[C@@H]1C(C)C', 'NS(=O)(=O)/C(Cc1ccccc1)c1cccc([N+](=O)[O-])c1', 'NS(=O)(=O)/C(CNC(=O)c1ccc(Cl)cc1)c1ccc(C(=O)[O-])cc1', 'NS(=O)(=O)/C(C#N)=C/c1ccc(Oc2ccc(F)cc2)cc1', 'NS(=O)(=O)/C=C\\C(=O)c1ccc([N+](=O)[O-])cc1C(F)(F)F', 'NS(=O)(=O)/C=C\\c1cc(Cl)cc(Cl)c1', 'NS(=O)(=O)/C(C#Cc1ccccc1)c1ccc(Cl)cc1', 'NS(=O)(=O)/C(C#N)=C/c1ccc(-c2ccc(Cl)cc2)o1', 'NS(=O)(=O)/C=c1/C(C#N)c2cc(Cl)c(C(=O)N3CCOCC3)cc2nc1/C=N/C(=O)c1ccccc1O', 'NS(=O)(=O)Nc1ccc(C(=O)OC(C(C)C)n2ccnc2)cc1', 'NS(=O)(=O)N[C@H]1C=Cc2ccc(C(=O)N3CCCC3)cc2N1', 'N#CCc1ccc(OCC[NH+]2CCC(CN3C[C@H](C)O[C@H](C4CC4)C3)CC2)cc1', 'NS(=O)(=O)/N=C/C(=O)Nc1ccc(Cl)cc1', 'NS(=O)(=O)/N=C/c1ccccc1O', 'NS(=O)(=O)N(CCc1ccccc1)c1nc2ccc(Br)cn2n1', 'NS(=O)(=O)N(C)C1CCC(Oc2ccc(N(C)C)cc2)CC1', 'NS(=O)(=O)N(Cc1ccc[nH]1)C[C@@H]1CCCCO1', 'NS(=O)(=O)N(C1CCCCC1)[C@@H]1CCC(=O)O1', 'NS(=O)(=O)N(C[C@H]1CCCN1S(=O)(=O)c1ccccc1)c1ccccc1', 'NS(=O)(=O)/N=C\\C1=CCCC[NH2+]1', 'NS(=O)(=O)/N=C\\c1ccc(OCc2ccccc2)cc1', 'Nc1c(C(=O)N2CC[C@@H](c3ccco3)C2)cnn1C(C)C', 'Nc1c(C2CC2)nc2scc(-c3ccccc3)c2c1C', 'Nc1c(CN(C)C(=O)[C@H]2CCO[C@H]2C)cnn1-c1ccccc1', 'Nc1c(CSc2ccccc2Cl)cnn1-c1ccc(Cl)cc1', 'Nc1c(Cl)cc(/C=C/Cl)c(Cl)c1', 'Nc1c(/C=N/c2ccc(Cl)cc2)cc(Br)nc1CO', 'Nc1c(/N=C/c2ccc3c(c2)OCO3)ncnc1N[C@H]1CCS(=O)(=O)C1', 'N=Cc1ncc(CN(C)C)cc1S(=O)(=O)Nc1ccc(Oc2ccc(C)cc2)cc1', 'N=Cc1nc2ccc(N[C@H]3CC[C@@H]([NH+](CC)C)c3)cn2c1[O-]', 'N=Cc1ncoc1-c1ccccc1OC1CCN(CC(F)(F)F)CC1', 'N=Cc1ncnc2cc(-c3cc(C)cc(OC)c3)nn12', 'Nc1cn[nH]c1', 'Nc1cn[nH]c1CSc1n[nH]c(=O)[nH]1', 'Nc1cn[nH]c1NC(=O)[C@H]1Cc2ccccc2S1', 'Nc1cn[nH]c1[C@@H]1N=C(C2CC2)c2nc3ccc(Cl)cc3n21', 'Nc1cn[nH]c1-c1ccccc1Br', 'Nc1cn[nH]c1Sc1ccccc1', 'Nc1cn[nH]c1Br', 'O=C(/C=C/c1ccc(Cl)cc1)Nc1ncc[nH]1', 'O=C(/C=C/OCc1ccccc1O)c1ccccc1F', 'O=C(/C=C/O)c1ccccc1', 'O=C(/C=C/Oc1ccc(Cl)cc1Cl)NC[C@@H](Cc1c[nH]c2ccccc12)[NH+]1CCCCC1', 'O=C(/C=C/O[C@@H]1COc2ccccc2O1)C1=NN(c2ccccc2)SC1=O', 'O=C(C1CCCCC1)N(Cc1ccccn1)c1ccccc1', 'O=C(C1CC1)N/C=C/c1ccnc2ccccc12', 'O=C(C1CC=C(c2ccc(Br)cc2)SC1)N1CCN(C(=O)COC(F)(F)F)CC1', 'O=C(C1CCN(C(=O)CN2CCOCC2)CC1)Nc1ccc([N+](=O)[O-])cc1', 'O=C(C1CC[NH+](Cc2ccccc2F)CC1)N1CCc2ccccc2[C@H]1c1ccccc1', 'O=C(CS(=O)(=O)Cc1cccc(Cl)c1)Nc1ccccc1C(=O)[O-]', 'O=C(/C(c1cccc(Cl)c1)N1CCOCC1)c1ccc(C(F)(F)F)cc1', 'O=C(CNc1ccc([N-]S(=O)(=O)c2ccccc2)c([N+](=O)[O-])c1)Nc1ccc(Cl)cc1Cl', 'O=C(CNc1ncon1)N[C@H]([C@@H]1CCS(=O)(=O)C1)c1ccccc1', 'O=C(/N=c1/o/[n+](=N/C(=O)OCc2ccccc2)nc1-c1cccs1)c1ccc(Cl)cc1', 'O=C(/N=c1/occn1Cc1ccccc1)c1cccc(F)c1', 'O=C(/C(CO)(c1ccccc1)c1cc2ccccc2o1)c1ccccc1', 'O=C(/C(COc1ccccc1)c1ccccc1)[C@@H]1COc2ccccc2O1', 'O=C(/C(c1cccs1)N1CCN(c2ccccc2OCc2c[nH]c3ccccc23)CC1)c1ccc2ccccc2c1', 'O=C(/C(c1cccs1)C(=O)c1cccs1)c1ccc(OC(F)(F)F)cc1', 'O=C(/C([O-])=N/S(=O)(=O)c1ccccc1Br)c1[nH]ccc1C', 'O=C(/C(c1cccs1)=[NH+]/Nc1cccc(C(F)(F)F)c1)N1CCN(c2cccc(Cl)c2)CC1', 'O=C(COc1ccc(Cl)cc1)Nc1ccc(F)c(C(F)(F)F)c1', 'O=C(CS(=O)(=O)Cc1cccc(Br)c1)N1CCN(c2ccccc2)CC1', 'O=C(COc1cc(Cl)ccc1C(=O)[O-])N1CCc2ccccc2C1', 'O=C(COc1ccc2ccccc2c1)NCC#CCOc1ccc(F)cc1', 'O=C(CSCCN1CCOCC1)c1cc(Cl)c(Cl)cc1Cl', 'O=C(CSCCO)N1CCC[C@H]1c1cccc(Cl)c1', 'O=C(CSCC(F)(F)F)N1CCC[C@](N2C(=O)c3ccccc3C2)C1', 'O=C(CSCC1CCCCC1)N1CCC[C@@H](c2nc3ccccc3n2C2CC2)C1', 'O=C(COc1ccc(F)cc1)NNC(=O)c1ccccc1Br', 'O=C(COc1ccccc1Cl)N1CCSCc2ccccc21', 'O=C(COc1ccc2c(c1)CCC(=O)N2)N1CCN(c2ccccc2)CC1', 'O=C(c1c[nH]nc1C(F)(F)F)N1CCC[C@@H]1CO', 'O=C(c1c[nH]nc1C1CC1)N1CCC[C@H](N2CCN(C3(c4ccccc4)CCO3)CC2)C1', 'O=C(c1c[nH]nc1CN1CC[NH+](Cc2ccccc2)CC1)N1Cc2ccccc2C1', 'O=C(c1c[nH]nc1C[C@H]1CN2CCCC[C@@H]2CO1)NCc1nc2ccccc2c(=O)[nH]1', 'O=C(c1c[nH]nc1C[NH+]1CCN(C(=O)[C@@H]2CCN(c3ccccc3F)CC2)C1)N1CCCCC1', 'O=C(/N=c1/[nH]cccc1Nc1ncccn1)c1cccs1', 'O=C(/C=C1CSCCS1)Nc1ccccc1Cc1c[nH]c2ccccc12', 'O=C(/C=C1CC(C)(C)[NH+](Cc2ccccc2)C1=O)c1cc(Cl)c(Cl)cc1Cl', 'O=C(/C=C1C(=O)N(c2ccc(Cl)cc2)c2ccccc21)c1ccccc1', 'O=C(CNS(=O)(=O)c1ccc(Cl)s1)Nc1cc(Cl)c(Cl)cc1Cl', 'O=C(C1CC[NH+](C2CCCC2)[C@@H](C(=O)NC2CCCCCCCCC2)C1)c1ccccc1', 'O=[N+]([O-])c1ccnc2ccccc12', 'O=[N+]([O-])c1ccnc2c1CCC[C@@H]2Nc1ccc(Cl)c(F)c1', 'O=[N+]([O-])c1ccnc2c1c(=O)n(Cc1cccnc1)[C@H](c1ccccc1Cl)c1ccccc1N2', 'O=[N+]([O-])c1ccnc2c1OCC[C@H]2NC(=O)[C@H]1CCCc2ccccc21', 'O=[N+]([O-])c1ccnc2c1NCC2', 'O=[N+]([O-])c1ccnc2c1[C@@H](N[C@@H]1CCOC1)CS2', 'O=[N+]([O-])c1ccnc2c1nnn2-c1cccc(C(F)(F)F)c1', 'O=[N+]([O-])c1ccnc2c1[C@H](NCc1ccco1)C2', 'O=[N+]([O-])c1ccnc2c1[nH]c(=S)n2C1CCCCCC1', 'OC[C@@]1([C@H]O)NCc2ccc(Br)cc21', 'OC[C@@]1([C@H]([NH3+])Cc2ccccc2)CCOC1', 'OC[C@@]1([C@H]2CCC[NH2+]C2)CCC[NH2+]C1', 'O=C([C@H](Nc1ccc(F)cc1)C(=O)N1CCN(c2ccccc2)CC1)N1CCCC1', 'O=C([C@H](N[C@@H](C(=O)[O-])C1CCCC1)Nc1cccc(Cl)c1)OCCO', 'O=C([C@H](N[C@@H]1CCS(=O)(=O)C1)N1CCCC1)Nc1nccs1', 'NS(=O)(=O)Cc1noc(-c2ccc(F)cc2)n1', 'NS(=O)(=O)Cc1nc(Oc2ccc(Cl)cc2)c2c(-c3cccc(Cl)c3)ccc([O-])c2n1', 'NS(=O)(=O)Cc1nnc(-c2ccc(N3CCCC3)cc2)n1N', 'NS(=O)(=O)C[C@@H]([N+](=O)[O-])c1cccc2ccccc12', 'NS(=O)(=O)C[C@@H](CNc1nc2ccc(Cl)cc2s1)N[C@@H]1CCS(=O)(=O)C1', 'NS(=O)(=O)C[C@@H](c1ccccc1)N1Cc2ccccc2C1', 'NS(=O)(=O)C[C@@H](O)CN1CCN(C[C@H](O)C(F)(F)F)CC1', 'NS(=O)(=O)C[C@@H](F)c1cccnc1', 'NS(=O)(=O)C[C@@H]([NH+]1CCc2ccccc2C1)c1ccccc1', 'NS(=O)(=O)C[C@@H](Br)c1ccccc1', 'NS(=O)(=O)C[C@@H]([NH3+])c1cccc(Cl)c1', 'O=C(c1cn[nH]c1)N(Cc1cccc2ccccc12)[C@@H]1CCS(=O)(=O)C1', 'O=C(c1cn[nH]n1)N1CCC(Oc2ccccc2)CC1', 'C[C@@H](c1ccoc1)n1c(N)nc2cccnc21', 'C[C@@H](c1ccsc1)N(C)C(=O)c1coc(C2CC2)n1', 'C[C@@H](c1cccc(C#N)c1)N(C)C(=O)CNC(=O)[O-]', 'C[C@@H](c1ccncc1)C(=O)N1CCCC(Cn2cc(Cl)cn2)C1', 'COCc1cc(OC)ccc1NC(=O)N(C)Cc1ccc(F)cc1', 'COCc1nc(C)ccc1C(=O)O[C@@H](CC)c1ccccc1', 'COC[C@@H](C)NC(=O)c1nn(-c2ccccc2)nc1C', 'COC[C@@H]1CN(C(=O)C2=c3ccccc3=[NH+]C2)N=N1', 'C=CCCC[C@@H](C)OC(=O)c1ccc2c(c1)OCO2', 'C=CCC(C)(C)C[C@](C)(O)CC', 'C=CCC1CC[NH+](Cc2cnc3[nH]c[nH+]c3c2)CC1', 'C=CCC[S@@](=O)Cc1nc2ccccc2n1C', 'C=CCCOc1ccc(N2C(=O)/C(=C/c3cc(C)on3)C2=O)cc1', 'C=CCC[C@@H](C)OC(=O)c1ccc(-c2nc(C)c3ccccc3n2)s1', 'C=CCCn1/c(=N/C(=O)c2ccccc2)sc2cc(C)c(C)cc21', 'C=CCC[C@H]1CCC[NH+](Cc2cccc(NC(C)=O)c2)C1', 'C=CCC[NH+](CC(C)C)[C@@H](C)C(=O)[O-]', 'C=CCC[C@@]1(Cc2cccc(OC)c2)CCCN(c2ccc(N(C)C)cc2)C1', 'Oc1cc(O[C@@H](Cc2ccc(Br)s2)C(F)(F)F)c(OC[C@H]2CCCO2)cc1', 'Oc1cc(O[C@@H](c2ccccc2)C(F)(F)F)ccc1Cl', 'Oc1cc(OCC(=O)N2CCCCC[C@@H]2c2ccc3c(c2)OCO3)cc(OCC(F)F)c1', 'Oc1cc(OCCOCOc2ccccc2)ccn1', 'Oc1cc(OCCn2cnc(-c3ccccc3)n2)cc(Cl)c1OCc1cccs1', 'Oc1cc(OCCSc2ncc(-c3ccccc3)n2Cc2ccccc2)ccc1Oc1ccccc1', 'Oc1cc(OCC[NH+]2CCCCC2)n[nH]1', 'Clc1cccc2nc(-c3cc(F)c(F)cc3Br)[nH]c12', 'Clc1cccc2nnnc2c1COc1cccc2c1OCO2', 'N#Cc1cc(Br)cc(NC[C@@H](O)COc2ccc(Br)cc2)c1', 'N#Cc1ccc(-c2n[nH]cc2-c2cccc(C(F)(F)F)c2)cc1', 'N#Cc1n[nH]nc1C[NH+]1CCC[C@@H]1c1cccc(Cl)c1', 'O=C(c1cnn(-c2ccccc2Cl)c1)N1CCC(C(F)(F)F)CC1', 'O=C(c1cnn(Br)c1)c1ccc(Cl)cc1', 'O=C(c1cnn(Cc2ccccc2)n1)N1CCN(Cc2ccccc2)CC1', 'O=C(c1cn2ccsc2n1)N1CCC(N2CCc3oncc3C2)CC1', 'O=C(c1cncc(I)c1)[C@@H]1CN2CCC[C@@H]2CO1', 'O=C(c1cnccc1Cl)N1CCSc2cc(Cl)ccc21', 'O=C(CN(CC1CCCC1)c1ccc(F)cc1)NCC1CC1', 'O=C(CN(C(=O)c1cccc(Cl)c1)N1CCCC1)c1ccccc1', 'O=C(CN(Cc1ccccc1)c1ccccc1)Nc1cc(Cl)ccc1Cl', 'O=C(CN(C1CCCCC1)c1ccccc1)N(C1CCCCC1)S(=O)(=O)c1ccccc1', 'O=C(CN(C[C@@H]1CCCO1)c1ccc(Cl)cc1)N1CCOCC1', 'O=C(CN(C[C@H]1CCCO1)c1ccccc1)N[C@H](c1ccccc1)c1ccccc1', 'O=C(CS(=O)(=O)C[C@@H]1CN(Cc2ccccc2)CCO1)Nc1cccc2ccccc12', 'O=C(CC(CC1(Cl)CC1)NCCCS(C)(=O)=O)C(=O)Nc1ccccc1', 'O=C(CC(c1ccccc1)N(C1CC1)c1ccccc1)N1CC[C@@]2(CCC(=O)Nc3ccccc32)C1', 'O=C(CC(=O)N1CCOCCN(S(=O)(=O)c2ccc(Cl)cc2)C1)N1CCOCC1', 'O=C(CC(Nc1ccccc1)N1CCOCC1)N[C@@H](c1ccc([N+](=O)[O-])cc1)C(F)(F)F', 'O=C(CC(F)(F)F)N[C@@H]1C=C[C@@H](C(=O)[O-])C1', 'O=C(c1c[nH]cn1)N[C@H](CO)c1ccccc1F', 'O=C(c1c[nH]c2nc[nH]c12)c1ccc(Br)cc1', 'O=C(c1cn[nH]n1)N(c1c(Cl)cccc1Cl)N1CCSCC1', 'O=C([C@H](c1ccccc1Br)N1CCOCC1)N1CCC(c2ccncc2)(C(=O)[O-])CC1', 'O=C([C@H](CSC(C)=O)c1ccc(F)cc1)N1CCc2ccc(F)cc2C1', 'O=C([C@H](CS)c1cc(F)ccc1F)N1CCC(OC2CC2)CC1', 'O=C([C@H](CSc1ccccc1)NCc1ccccc1)c1ccccc1', 'O=C([C@H](c1c([N+](=O)[O-])cccc1[N+](=O)[O-])N1CCSCC1)N1CCCC[C@H]1C1CC1', 'O=C([C@H](c1c(Cl)cc([O-])cn1)N1CCCCC1)N1CCN(Cc2ccco2)CC1', 'O=C([C@H](c1c(F)cccc1F)Oc1cccc(C(N)=O)c1)N1CCN(c2ccccc2F)CC1', 'O=C(c1c[nH]c2cc(Cl)ccc12)c1ccc(Br)cc1', 'O=C(c1c[nH]c2ccccc12)N1CCC[C@H]1CN1CCc2ccccc21', 'O=C(c1c[nH]cn1)O[C@@H]1CCN(c2cccc(F)c2)C1', 'O=C(c1c[nH]cn1)[C@H]1CCS(=O)(=O)C1', 'O=C(c1c[nH]cn1)N1CCC(Oc2ccc(C(=O)NCc3cccs3)cc2)CC1', 'O=C(/C(Cc1ccccc1)c1ccc(F)cc1)c1c(F)cccc1F', 'O=C(/C(Cc1ccccc1)Nc1nccs1)c1ccc(F)cc1', 'O=C(/C(c1cccs1)Cn1ncc2ccccc2c1=O)c1cccs1', 'O=C(/C(c1cccs1)(C(=O)[O-])C1CCN(S(=O)(=O)c2ccc([N+](=O)[O-])cc2)CC1)/c1ccccc1', 'O=C(/C(c1cccs1)c1ccc(Br)cc1)c1ncc[nH]1', 'O=C(/C(c1cccs1)=C/c1ccc(O)cc1)c1ccc(S(=O)(=O)N2CCCC2)cc1', 'O=C(/C(c1cccs1)[C@@H]1CC=CC=C1O)c1ccccc1O', 'O=C(/C(c1cccs1)S(=O)(=O)c1ccccc1)[C@@H]1CC(=O)N(c2ccccc2)[C@H]1Cc1ccccc1', 'O=C(/C(c1cccs1)[NH+]1CCCCC1)c1cn(Cc2ccccc2)nn1', 'O[C@@H]1Cc2ccccc2CN1C(=O)NCC[C@@H](O)c1ccccc1', 'O[C@@H]1C[C@@](O)(/N=C/c2cccn2Cc2ccc(Cl)cc2Cl)CC1', 'O[C@@H]1C[C@@]1(O)CCc1ccc(Cl)cc1', 'O[C@@H]1C[C@@]2(OC(=O)C[NH+]3CCCC[C@@H]23)C1', 'Oc1cc(Br)cc(F)c1N1CCN(c2[nH+]cccc2[N+](=O)[O-])CC1', 'Oc1cc(Br)ccc1-c1cccc2ccccc12', 'Oc1cc(Br)cc2cc(F)ccc12', 'Oc1cc(C(F)(F)F)ccc1NCc1ccc(N2CCCC2)cc1', 'Oc1cc(CCCn2ccc(CO)n2)ccc1CO', 'Oc1cc(CCOc2ccccc2F)c2cccnc2n1', 'Oc1cc(CCNCc2nc3cc(Cl)ccc3s2)ccc1F', 'Oc1cc(CCn2nc(-c3cc(F)ccc3F)nn2)ccc1Cl', 'Oc1cc(CC[NH+]2CCC[C@H]2c2ccco2)ccc1N1CCOCC1', 'Oc1cc(CC[NH2+][C@H](c2ccccn2)c2ccco2)ccc1F', 'Oc1ccc(F)cc1NC(=O)Cc1ccc(F)c(Cl)c1', 'Oc1ccc(F)cc1N(CCCC#N)C1CC1', 'Oc1ccc(F)cc1Nc1cccc(C(=O)N2CCOCC2)c1', 'Oc1ccc(F)cc1N1CCOC[C@H]1C(=O)N(Cc1ccccc1)C1CC1', 'Oc1ccc(F)cc1N[C@@H]1CCCSC1=O', 'Oc1ccc(F)cc1NS(=O)(=O)c1ccccc1', 'Oc1ccc(F)cc1N[C@H](CO)c1ccc(F)cc1', 'Oc1ccc([N+](=O)c2ccccc2)cc1OSc1ccccc1', 'Oc1ccc([N+](=O)c2occc2Cl)cc1O', 'Oc1ccc([N+](=O)c2onc(C3CC3)n2)cc1OC[C@@H]1CCCO1', 'Oc1ccc(Oc2ccc(NCc3ccco3)cc2)c(Cl)c1', 'Oc1ccc(Oc2ccc(NC(=O)Nc3cc(Cl)ccc3O)cc2)cc1F', 'Oc1ccc(Oc2ncn[nH]2)cc1', 'Oc1ccc(Oc2ccc(Cl)cc2)c(O)c1', 'Oc1ccc(Oc2cnc(C[NH+]3CCCCC3)s2)cc1', 'Oc1ccc([C@H](CNC(=O)c2cnc([C@H]3CCCCO3)n2C2CC2)[NH+]2CCCC2)s1', 'Oc1ccc([C@H](c2cc3ccccc3[nH]c2=O)c2cc[nH]n2)cc1', 'Oc1ccc([C@H](O)[C@@H](NC(=O)NC2CCCC2)c2cccs2)cc1', 'Oc1ccc([C@H](Nc2cc(C(F)(F)F)c3ccccc3n2)c2ccncc2)cc1', 'Oc1ccc([C@H]([O-])CCOc2ccc(Cl)cc2)cc1', 'Oc1ccc([C@H](/N=C2\\Oc3ccccc3C2=O)c2ccccc2Cl)cc1OCc1ccccc1', 'Oc1ccc([C@H]([NH2+]C[C@H]2CCCO2)[C@@H]2CCSc3ccccc32)cc1', 'Oc1cc(-n2nnc(C(F)(F)F)n2)ccc1-c1nc2ccccc2s1', 'Oc1cc(-n2nnc(-n3cccn3)n2)ccc1F', 'Oc1cc(-n2nnc(/C=C/c3ccccc3)c2N2CCCCC2)cc(Br)c1F', 'Oc1cc(Br)cn/c(=[N+]([O-])c2ccccc2)c1', 'Oc1cc(Br)cnc1N[C@@H]1CC[NH2+]C1', 'O=C([C@H](Sc1nnnn1Cc1ccccc1)[C@H]1CCCO1)N1C[C@H](CO)CCO1', 'O=[N+]([O-])c1c(Br)scc1C(=O)NCCNC(=O)c1ccccc1F', 'O=[N+]([O-])c1c(Cl)nn(C2CC2)c1NCc1csc2ccccc12', 'O=[N+]([O-])c1c(Cl)nnc2[nH]nc(-c3ccc(F)c(F)c3)c12', 'O=[N+]([O-])c1c(F)cc2[nH]ccc2c1Cl', 'O=[N+]([O-])c1c(F)cc2nc(N3CCC(O)CC3)n[nH]c12', 'O=[N+]([O-])c1c(Br)scc1C(=O)Nc1cccs1', 'O=[N+]([O-])c1c(Br)scc1Oc1cccc2cccnc12', 'O=[N+]([O-])c1c(Br)scc1N1CCn2c1nn2-c1ccc(Br)cc1', 'O=[N+]([O-])c1c(Br)scc1[C@@H]1Nc2ccc(Br)cc2N1', 'O=[N+]([O-])c1c(Br)scc1[C@H](O)CS(=O)(=O)c1ccccc1', 'O=C(c1cn2ccsc2n1)N1CC(c2ccccc2)=N1', 'O=C([O-])/C(=C/c1ccccc1)NC[C@@H]1CCCO1', 'O=C([O-])/C(C[C@@H](O)c1ccccc1)N(Cc1nc2ccccc2n1Cc1ccccc1)C1CC1', 'O=C([O-])c1[nH]ncc1C(=O)N1CCN(c2ccc(C(=O)[O-])cc2)CC1', 'O=C([O-])/C=C/c1ccc(-c2ccccc2[N+](=O)[O-])cc1O', 'O=C([O-])c1nc(CSc2cc(F)ccc2F)c(F)cc1F', 'O=C([O-])c1nn(-c2ccccc2)cc1N1CCN(Cc2ccccc2)CC1', 'O=C([O-])/C(=C/c1ccccc1)c1ccc(O)cc1', 'O=C([O-])/C(=N/c1ccc2c(c1)OCO2)N1CCC(Oc2ccccc2)CC1', 'O=C([O-])c1csc(C[NH2+][C@H](c2ccccc2)C2CC2)n1', 'O=C([O-])c1c(N2CC(c3ccccc3)n3nc(C4CCCCC4)ncn23)coc1-c1noc2c1CCCC2', 'O=C([O-])c1ccccc1C[C@@]1(O)C[NH2+]CCO1', 'O=C([O-])c1cnc(S[C@@H]2CCS(=O)(=O)C2)n1-c1ccc(Cl)cc1', 'Oc1ccc(F)c(/C=C2\\N=c3ccccc3=[NH+][C@@H]2c2cccc([N+](=O)[O-])c2C(F)(F)F)c1', 'Oc1ccc(F)c(/N=C/c2ccn(-c3ccccc3)n2)c1', 'Oc1ccccc1[C@@H]1CCCN1C(=O)NCCCCn1cncn1', 'Oc1ccccc1[C@@H]1OCC[NH+](CN2C(=O)c3ccccc3C2=O)C1', 'Oc1ccccc1[C@@H]1[NH+]=c2ccccc2=[NH+][C@@H]1c1cccc(Cl)c1', 'N=C1\\[C@@H](c2cccc(F)c2)NC(=S)NC1=O', 'N[C@@H](NCC1(C)CC1)C(=O)N1CCC(C(F)(F)F)CC1', 'N[C@@H](NC(=O)[C@H]1CCC[NH+](C2CC2)C1)C1(Cn2cncn2)CC1', 'N[C@@H](NCc1c[nH]nc1-c1ccccc1)C(=O)N1CC[C@H]2[C@@H](CCCc3ccc(F)cc32)C1', 'N[C@@H](NC1CCN(c2ccccc2)CC1)c1ccccc1', 'N[C@@H](NC[C@H](c1ccco1)[NH+](C)C)c1ccccc1', 'ClCc1cnn[nH]1', 'ClCc1cnn(-c2ccccc2)c1N', 'O=[N+]([O-])c1c(-c2c[nH]c[nH+]2)[nH]c2c1cnn2-c1cccc(Br)c1', 'O=[N+]([O-])c1c(-c2c[nH]nc2Cc2ccccc2)oc2ccccc12', 'O=[N+]([O-])c1c(OCCN2CCCCCC2)cnn1C(Cl)F', 'O=[N+]([O-])c1c(OCc2nccn2Cc2ccccc2)oc2cc(Cl)cc(Cl)c12', 'O=[N+]([O-])c1c(OCOC[C@H]2CCCO2)c(Cl)ccc1F', 'O=[N+]([O-])c1c(OC[C@H]2CCCO2)ccnc1Sc1ncccn1', 'ClCc1csc2s1CCCC2', 'ClCc1csc2nc(-c3ccccc3)nn12', 'OC[C@@H](CC1CCOCC1)NNC(=O)[C@@H]1CCCO1', 'OC[C@@H](C(=O)N1CCc2ccccc2C1)N1CCc2ccccc2C1', 'OC[C@@H](Cc1cc(F)cc(Br)c1)OCC(F)(F)F', 'OC[C@@H](C1CCSCC1)N1CCC[C@@H](c2cc(-c3ccccc3)no2)C1', 'OC[C@@H](CO)N1CCC(c2ncc[nH]2)CC1', 'OC[C@@H](CNc1ccnc(Cl)c1)N[C@@H]1CC(CO)(c2ccccc2)n2ccccc2=N1', 'OC[C@@H](C[C@@H]1CCCO1)N(CC(=O)Nc1ccccc1)c1ccccn1', 'OC[C@@H](Cn1cc(Cl)cn1)NCc1ccccc1', 'O=C(c1cc(F)ccc1C(=O)c1ccc(Cl)s1)N1CCCCC1', 'O=C(c1cccs1)N(Cc1ccc[nH]1)c1nc(N2CCOCC2)nc2cccnc12', 'O=C(c1cnn2c1CCCC2)N1CCO[C@H](c2ccc([N+](=O)[O-])cc2)C1', 'O=C(c1cnn2nc(C3CC3)oc12)N1CCC([C@H]2CCOC2)CC1', 'O=C(c1cc([S@](=O)C23CC4CC(CC(C4)C2)C3)oc1-n1cnnn1)c1ccccc1', 'Clc1ccsc1C(=O)N[C@@H](c1ccccc1)c1ccccc1O', 'Clc1ccsc1C[N-]c1ccc(C(F)(F)F)nc1', 'Clc1ccsc1C1CC[NH+]([C@@H](C(=O)[O-])C(F)(F)F)CC1', 'Clc1ccsc1CN(Cc1cccc(F)c1)c1ccccc1', 'Clc1ccsc1C[NH+]1CCC[C@H]1Cn1cccn1', 'Clc1ccsc1C[NH2+][C@@H]1CCc2c(F)cccc21', 'Oc1ccc(O[C@@H](Cc2ccc(Cl)cc2)c2cccc(Cl)c2)cc1F', 'Oc1ccc(O[C@@H]([NH3+])C[C@@H]2OCc3ccccc32)cc1', 'Oc1ccc(O[C@@H](c2ccco2)C(=O)NCc2ccccc2)cc1', 'Oc1ccc([N+](=O)[O-])c(C(=O)c2ccc(NCCO)cc2)c1', 'Oc1ccc([N+](=O)[O-])c(I)n1', 'Oc1ccc([N+](=O)[O-])c(NN=C2CCCCC2)c1', 'Oc1ccc([N+](=O)[O-])c(-c2ccc(F)cc2F)c1', 'Oc1ccc([N+](=O)[O-])c(S(=O)(=O)NCc2c(O)cccc2O)c1', 'Oc1ccc([N+](=O)[O-])c(Cl)c1Oc1cccc2ccccc12', 'Oc1ccc([N+](=O)[O-])c(/C=C/Nc2ccccc2Br)c1', 'Oc1cc(-n2cnc(CN3CCOCC3)n2)ccc1F', 'Oc1cc(-n2cncn2)ccc1CNC1=Cc2ccccc2OC1', 'Oc1cc(F)c(C(=O)NCC2(c3ccccc3)CCOCC2)cc1Cl', 'Oc1cc(F)c(Oc2ccc(Cl)cc2F)c(F)c1', 'Oc1cc(F)c(Nc2ccc(Br)cc2)cc1Cl', 'Oc1cc(F)c([C@@H](O)CN2CCCCCC2)c(Cl)c1', 'Oc1cc(F)c(-c2cccnc2)nc1CO', 'Oc1cc(F)c(S(=O)(=O)[N-]c2ccc3c(c2)OCCO3)cc1C(=O)[O-]', 'Oc1cc(F)c(Cl)cc1N1CC[C@H](N2CCCC2)C1', 'Oc1cc(F)c([O-])c(C(=O)O[C@@H]2CCCCC2(Cl)F)c1Cl', 'Oc1cc(F)c([C@H](O)C(F)(F)F)cc1', 'Oc1cc(F)c(Br)cn1', 'Oc1cc(F)c(/C=N/NS(=O)(=O)c2ccccc2)cc1Cl', 'O=C(CS(=O)(=O)c1cc(F)ccc1F)N[C@@H](CO)c1ccco1', 'O=C([C@H](CCC1CC1)c1c[nH]c2ncccc12)N1CCCC1', 'O=C([C@H](CC(F)(F)F)N1CCCS1(=O)[O-])N1CCc2ccccc21', 'O=C([C@H](CCc1cccnc1)NCc1ccccc1)N1CCCC[C@H]1c1ccccc1', 'O=C([C@H](CC1CC1)N1CCOc2ccccc2C1)OC[C@@H]1CCCCO1', 'O=C([C@H](CCO)Nc1cc(F)ccc1F)NC12CC3CC(CC(C3)C1)C2', 'O=C([C@H](CCSc1ccccc1)c1ccsc1)N1CCCCC1', 'C[C@@H](O)CN(C)[C@H]1CC[C@@H]([NH3+])C1', 'C[C@@H](O)c1cccc(CS[C@@H]2CCCOC2)c1', 'O[C@@H]1c2cc([N+](=O)[O-])ccc2Nc2cc(O)ccc21', 'O[C@@H]1c2cc(C(F)(F)F)ccc2N[C@@H]2C[C@H]1C[C@H]2O', 'O[C@@H]1c2cc(OC(F)(F)F)[nH]c2N[C@H]1C(=O)[O-]', 'O[C@@H]1c2cc(NC(=O)COCc3ccccc3)ccc2O[C@@H]1C(=O)N1CCCC1', 'O[C@@H]1c2cc(F)cc(Cl)c2C(=O)N1CCCCn1cccn1', 'O[C@@H]1c2cc(-c3ccco3)ccc2O[C@@H]1c1cccc(Br)c1', 'O[C@@H]1c2cc(S(=O)(=O)[O-])ccc2[C@H]1c1ccccc1', 'O[C@@H]1c2cc(Cl)ccc2N[C@H]1CC(F)(F)F', 'O[C@@H]1c2cc(Br)ccc2C[C@H]1Cl', 'COC(=O)c1ccc(Cl)c([O-])c1C(=O)Nc1ccc(OC)cc1', 'COCc1c(C)sc2occ(-c3ccc(OC(F)F)cc3)c12', 'COC(=O)[C@@H](C)n1nc(C)c2c(-c3ccccc3)nccc21', 'COC[C@H](O)CN(C)C(=O)[C@@H]1CCC(=O)NC1', 'COC[C@H](C)NC(=O)[C@@H]1C[C@H]1c1cc(Cl)ccc1Cl', 'COC[C@H](NCc1ccccc1C)[C@H]1CCOC1', 'COC[C@H]([NH3+])Cc1ccc(OC(F)F)cc1', 'Cc1ccsc1/C=C/C(=O)N(Cc1ccccc1)Cc1ncns1', 'Cc1ccc(OCc2noc([C@H]3CCCN(C(=O)[C@H]4C[C@H]3C(F)(F)F)n4C)c2C)cc1C', 'Cc1cccc2c1[C@@H](Cc1cccnc1)C2CC[NH3+]', 'Cc1ccc2nc(C)c(C(=O)OCC(=O)Nc3nc4ccc(Cl)cc4s3)nc2c1', 'N=C1CCCC1=CCc1c(O)cccc1Cl', 'N=C1C=C(C(=O)N[C@H](c2ccc(OCN3CCOCC3)cc2)c2cccs2)c1', 'N=C1CN(C(=O)[C@H]2CCCN(C(=O)C3COC3)C2)c2ccccc21', 'N=C1C[NH+](C2CC[NH+](CC(N)=O)CC2)CCN1Cc1ccccc1', 'N=C1C[C@]2(CCC[NH+](Cc3ccccc3)n2)Nc2cc(Cl)ccc2N1', 'COc1nnnn1CCC(=O)N(Cc1ccc[nH]1)[C@@H](C)c1ccccc1', 'COc1nnnc(NC(=O)CNc2ccc(Cl)c([N+](=O)[O-])c2)c1C', 'COc1nnnn1-c1cncc(NCc2ccc(C(F)(F)F)cc2Cl)c1', 'COc1nnnn1CCNC(=O)c1ccc(Cl)c([N+](=O)[O-])c1C', 'COc1nnnn1/N=C/c1ccc(C)cc1', 'COc1nnnn1[C@@H]1CC(C)(C)CCN1C(=O)c1cc(F)ccc1F', 'COc1nc2snc(/C=C(Cl)C3=NOCCO3)c(=O)c2cc1CN1C[C@H](C)O[C@H](C)C1', 'CCS(=O)(=O)CCN(C)C(=O)CCc1ccccc1Cl', 'CCc1c(C)sc(NC(=O)c2c3ncn(CC(C)C)c3sc2C)c1C#N', 'CCc1ccc(-c2csc(=S)n2C2CCCCC2)cc1', 'CCc1cnc(C[NH+]2CC[C@@H](C)C[C@H]2C)c(C)n1', 'CN(c1cc(Cl)cc(N)n1)C1CC[NH+]([C@@H](C)C(C)C)CC1', 'ClCc1cn(C2CCN(c3ncccn3)CC2)cn1', 'ClCc1cn(N[C@@H]2CCOC3(CCOCC3)C2)s1', 'OCS[C@@H](C1CCCCC1)C(c1ccccc1)c1ccccc1', 'OCS[C@@H]1N=c2ccccc2=[NH+]C1=O', 'Oc1ccc(F)cc1SC1CCN(C(=O)c2ccc(Cl)nc2)CC1', 'Oc1ccc(F)cc1S(=O)(=O)N(C1CC1)[C@@H]1CCS(=O)(=O)C1', 'Oc1ccc(F)cc1Sc1nc2ccc(C(F)(F)F)cc2s1', 'Oc1ccc(F)cc1S[C@@H]1CC[C@@H]([NH+]2c3ccccc32)C1', 'Oc1ccc(F)cc1S[C@H]1CCC[NH+](CC(F)F)C1', 'C[C@@H](NS(=O)(=O)c1ccc(C(=O)N2CCCC2)cc1)c1cccc(C#N)c1', 'C[C@@H](NCc1ccoc1Br)Cc1cccc(F)c1', 'C[C@@H](N[C@@H](C)C(=O)N(C)C)C(=O)C1=CN(C2CC2)C1', 'C[C@@H](N[C@@H](c1ccc(F)cc1)c1n[nH]c(N)n1)c1ccncc1', 'ClC[C@H]1OCCc2ccccc21', 'ClC[C@H]1Oc2ccccc2NC1', 'ClC[C@H]1O[C@H]2[C@@H](c3ccccc3)N=C12', 'ClC[C@H]1O[C@]2(CCSc3ccccc32)O1', 'ClC[C@H]1O[C@@](NC(=O)C2CCC2)Cc2ccc(Br)cc21', 'COCc1noc([C@@H]2CC(=O)c3[nH]nc(C)c3C2)n1', 'COCc1n[nH]c(C)c1[C@@H](C)NC(=O)c1ccc(OC)c(F)c1', 'COCc1nc(C(=O)N2N=CN(CC#N)CC2)cs1', 'COCc1nnc(NC(=O)CCc2c(C)cccc2C)s1', 'O=[N+]([O-])c1c(Cl)nsc1Nc1ccc([N+](=O)[O-])c([N+](=O)[O-])c1', 'O=[N+]([O-])c1c(F)cc([N+](=O)[O-])cc1OCCCCO', 'O=[N+]([O-])c1c(F)cc(C(=O)[O-])cc1F', 'O=[N+]([O-])c1c(F)cc(OC[C@H](O)COc2cccc(F)c2)cc1Cl', 'O=[N+]([O-])c1c(F)cc(NCCCSC2CC2)cc1', 'O=[N+]([O-])c1c(F)cc(F)cc1NCc1cnn2cnnc2c1', 'O=[N+]([O-])c1c(F)cc(Cl)cc1OCC(F)(F)F', 'O=[N+]([O-])c1c(F)cc(Br)cc1NC[C@H](O)C1CCC1', 'O=[N+]([O-])c1c(Cl)n2nnnc2n1Cc1ccc(F)cc1', 'O=[N+]([O-])c1c(-c2nnn[n-]2)sc2cc(Br)ccc12', 'O=[N+]([O-])c1cc([C@H]2CCCN(c3ncccn3)C2)c(F)c(Cl)c1', 'O=[N+]([O-])c1cc(Br)ccc1OCCN1CCC[NH2+]CC1', 'Oc1cccc([C@H]2CNc3ccccc32)c1F', 'Oc1cccc([C@H]2OCCN2C(=O)CSCc2ccccc2)c1', 'Oc1cccc([C@H]2[C@H]3[NH+]=c4ccccc4=C3CC[C@@H]2O)c1', 'OC[C@H]1[C@H]([NH2+]C2CCCCC2)CCCCC1(CCCC(F)(F)F)Nc1ccccc1', 'OC[C@H]1[C@H]2CCCC[C@H]2[C@H]1O', 'N#CCS\\C(=N\\OCCc1cccc(Cl)c1)c1ccccc1', 'N#CCS(=O)(=O)c1ccccc1C[NH+]1CCc2ccccc2C1', 'N#CCSCCCNC(=O)c1cccn1C(F)F', 'N#CCSc1cccc(OCc2ccccc2)c1', 'N#CCS[C@@H]1CCC[NH+](C2CCN(c3cc(Cl)ccc3)CC2)C1', 'Clc1nnnn1-c1c(Cl)cccc1Cl', 'Clc1nnnn1-c1ccc(O)cc1', 'Clc1nnnn1-c1cnccc1', 'Clc1nnnn1Cc1csc2ccccc12', 'Clc1nnnn1C1CC1', 'Clc1nnnn1c1ccc(OCC(N)=O)cc1', 'Clc1nnnn1/N=C/c1ccc(Br)cc1', 'Clc1nnns1\n\n\n', 'Clc1nnnn1c1[nH]c(C2CC2)nc1N1CCc2ccccc21', 'Clc1nnnn1c1cc(F)ccc1F', 'Clc1nnnn1c1nc(O)c(Cl)c([O-])c1Cl', 'Clc1nnn(-c2ncn[nH]2)c1CNC(=O)c1ccc[nH]1', 'Clc1nnn(-c2ncn(-c3cccc(Cl)c3)c2-c2ccn[nH]2)n1', 'Clc1nnn(-c2ncnc3sc4c(c23)CCC4)n1', 'Clc1nnnn1-c1cc(F)ccc1F', 'Clc1nnnn1CCCC[NH+]1CCC(Cc2ccccc2)CC1', 'Clc1nnnn1CCc1ccccc1', 'Clc1nnnn1CC1CCC1', 'Clc1nnnn1CCOC1CC1', 'Clc1nnnn1CCN1CCOCC[C@@H]1C[NH+]1CCCC1', 'Clc1nnnn1CC[C@@H]1CSc2ccccc21', 'Clc1nnnn1CCSc1nc(CC2CC2)n[nH]1', 'Clc1nnnn1CC[NH+]1CCC[C@H]1c1c[nH]cn1', 'Clc1nnnn1CC[NH2+]Cc1ccccc1OC(F)F', 'O=C(Cc1c[nH]c2ccccc12)NC1CCN(c2cccc[nH+]2)CC1', 'O=C(Cc1c[nH]nc1C(=O)NC[C@H]1CN2CCCC[C@@H]2CO1)Nc1nccs1', 'O=C(c1c[nH]c2nc[nH]c12)N1CCC[C@H]1C[C@H](O)c1ccccc1', 'O=C(c1c[nH]c2ncn(C(=O)c3ccccc3)c12)N1C[C@@H](COc2ccccc2F)C1', 'O=C(CS(=O)(=O)Cc1nnc(-c2ccoc2)o1)Nc1ccccc1-c1ccccc1', 'O=C(CNS(=O)(=O)c1ccc(Br)cc1)N1CCC[C@@H](c2nc3ccccc3s2)C1', 'O=C(CNS(=S)=N/O)c1ccccc1', 'Oc1cc(-n2cc(C(F)F)ccc2=O)ccc1C(=O)Nc1cc(F)ccc1[N+](=O)[O-]', 'Oc1cc(-n2cc(I)c([O-])n2)ccc1F', 'Oc1cc(-n2cc(F)cn2)cc(C(=O)Nc2ccc(F)cc2)c1', 'Oc1cc(-n2cc(-c3ccccc3)nn2)ccc1Br', 'Oc1cc(-n2cc(Br)cn2)ccc1NCc1ccc(F)cc1', 'Oc1cc(-n2nnnc2CN2CCN(c3ccccc3F)CC2)ccc1NC[C@@H]1CCCO1', 'Oc1cc(-n2nnnc2C2CC2)c[nH]1', 'Oc1cc(-n2nnnc2O)cs1', 'Oc1cc(-n2nnnc2N2CCN(c3ccc(Cl)c(Cl)c3)CC2)ncn1', 'Oc1cc(-n2nnnc2-c2c(F)cccc2F)c2ccccc2c1OCc1c(F)ccc(Br)c1', 'Oc1cc(-n2nnnc2SCC(=O)c2ccccc2)coc1=O', 'Oc1cc(COc2ccc(Cl)cc2)c(Cl)cc1Cl', 'O=C(c1cn(-c2ccccc2)nc1-c1ccccc1)N1CCN(C2CC2)CC1', 'O=C(c1cn2ccsc2c1)N1CCn2nc(-c3ccc(F)cc3)cc2[C@@H]1c1ccccc1', 'O=C(c1cn2ccsc2n1)N1CCC(Oc2cc(S(C)(=O)=O)nc3ccccc23)CC1', 'O=C(c1cn(-c2ccccc2)nn1)O[C@@H](c1cccs1)C1CC1', 'O=C(c1cn(-c2ncccc2Cl)o1)N1CCOCC1', 'O=C(c1c[nH]nc1NC[C@H](c1ccco1)N1CCc2ccccc21)N1CCOCC1', 'O=C(c1c[nH]nc1N(Cc1ccccc1)Cc1ccc(Cl)cc1)C1CC1', 'O=C(c1c[nH]nc1Nc1ccc(C(=O)NC2CC2)cc1)c1ccccc1', 'O=C(c1c[nH]nc1N1CCOCC1)N1CCN(Cc2cccc(C(F)(F)F)c2)CC1', 'O=C(c1c[nH]nc1N[C@H]1CCSc2ccccc21)N1CC=C(c2ccccc2)CC1', 'O=C(c1cn[nH]c1-c1ccc(Cl)cc1)N1CCN(C(=O)N[C@@H]2CCSc3ccccc32)CC1', 'O=C(c1cnc(CN2CCN(c3ncccc3F)CC2)s1)c1ccccc1', 'O=C(c1cnc(OCCc2ccccc2)cc1)N1CCC[C@@H](OCc2ccccc2F)C1', 'O=C(c1cnc(N2CCN(CC3CC3)CC2)cn1)N1CCOCC1', 'O=C(c1cnc([C@@H]2CCCC[C@@H]2C(=O)N2CCCC2)s1)N1CCOCC1', 'O=C(c1cnc(-c2ccccc2)o1)N(Cc1ccccc1F)[C@H]1CCS(=O)(=O)C1', 'O=C(c1cnc(SCc2ccc(F)cc2)s1)N1CC[C@H]2CCCC[C@@H]21', 'O=C(c1cnc(/N=C/CCO)cc1)OCc1nc(-c2ccncc2)no1', 'O=C(CS(=O)(=O)Cc1cccc(F)c1)Nc1ccccc1OCc1ccccc1', 'O=C(CS(=O)(=O)[N-]c1ccc(OC(=O)c2cc(F)ccc2F)cc1)N1CC[C@H]([NH+]2CCCC2)C1', 'O=C(CS(=O)(=O)c1ccc2ccccc2c1)Nc1ccccc1', 'O=C(CS(=O)(=O)[C@@H](c1ccccc1)N1CCOCC1)Nc1ccc2c(c1)OCCO2', 'O=C(CS(=O)(=O)[C@H]1CCCC[C@@H]1[NH2+]C1CC1)N1CCC(CC(N)=O)CC1', 'O=C(/N=C\\c1ccco1)[C@@H]1CCCc2sccc21', 'O=[N+]([O-])c1c(Cl)nsc1C(=O)NC1(CC2(O)CCCC2)CC1', 'O=C(c1c[nH]c(Nc2cccc(F)c2)n1)N1CCC[C@@H](C(F)(F)F)C1', 'O=C(c1c[nH]c(N2CCCCC2)n1)N1CC[C@H](n2cncn2)[C@@H]1C#N', 'O=C(c1c[nH]c(NN=C(c2ccccc2)c2ccccc2)n1)NNc1ccc(F)cc1', 'O=C(c1c[nH]c(N/N=C/c2cccc(Cl)c2F)n1)OCc1ccncc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnc2[nH]cnc12', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1-c1ccc(F)cc1', 'O=[N+]([O-])c1c(Cl)n2nnc(-c3ccccc3)c2cc1O', 'O=[N+]([O-])c1c(Cl)n2nnnc2n1-c1ccccc1', 'O=[N+]([O-])c1c(-c2ccsc2)ncnc1NCc1ccccn1', 'O=[N+]([O-])c1c(-c2cc(-c3ccc(F)cc3)n[nH]2)ncnc1C1CC1', 'O=[N+]([O-])c1c(-c2ccccc2)sc(SCC(=O)N2CCOCC2)n1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnnc1N', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1-c1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn2cccc12', 'O=[N+]([O-])c1c(Cl)nn(-c2ccccc2)c1Nc1ccc([N+](=O)[O-])cc1', 'O=[N+]([O-])c1c(Cl)nn(CC(=O)N2CCO[C@@H](Cn3ccc(Br)c3)C2)c1C1CC1', 'O=[N+]([O-])c1c(Cl)nnc(NC2(C#N)CCOCC2)c(-c2ccc(Cl)cc2)c1=O', 'O=[N+]([O-])c1c(Cl)nnc2cc(-c3ccccc3)n(-c3ccc(F)cc3)c12', 'Oc1ccccc1OSCc1noc(-c2ccccc2)n1', 'Oc1ccccc1OS(=O)(=O)c1ccccc1', 'Oc1ccccc1OS[C@H](c1ccccc1)N1CC[C@@H]([N+](=O)[O-])C1', 'Oc1cccc(SCCc2nnc(-c3ccccc3[N+](=O)[O-])o2)c1', 'Oc1cccc(SC2=[N+](c3ccccc3)Nc3cc(F)c(F)cc32)c1', 'Oc1cccc(SCn2cnc3ccccc3c2=O)c1', 'Oc1cccc(SCS(=O)(=O)c2ccsc2)c1', 'Oc1cccc(SC[C@H]2C[C@H]2c2ccc(Cl)cc2)c1', 'Oc1ccccc1[C@H](C[NH2+][C@@H](CO)c1ccc(Cl)cc1)c1ccccc1', 'Oc1ccccc1[C@H](c1ccccc1)N1CCC(C[NH2+][C@@H](CO)C(F)(F)F)CC1', 'Oc1ccccc1[C@H](O)[C@@H]1CCc2ccccc2C1', 'Oc1ccccc1[C@H](NCc1nc(-c2ccccc2)n[nH]1)c1ccc(Cl)cc1', 'Oc1ccccc1[C@H]([C@@H](O)c1ccncc1)NC1CC1', 'Oc1ccccc1[C@H](Cl)c1ccc[nH]c1=O', 'Oc1ccccc1[C@H]([NH2+]C[C@@H]1CC2(CCCC2)O1)c1ccccc1', 'Oc1ccc(Oc2ncccc2C2(C[NH2+]C(F)(F)F)CC2)cc1', 'Oc1ccc([N+](=O)COCc2ccco2)cc1Cl', 'Oc1ccc([N+](=O)COc2ccccc2C(=O)[O-])cc1', 'Oc1ccc(O[C@H](C(=O)NNC(=O)c2nc(C3CC3)c3ccccn23)c2ccccc2)cc1', 'Oc1ccc(O[C@H](Oc2ccccc2)N2CCN(c3ccccc3)CC2)c(Cl)c1', 'Oc1ccc(O[C@H](F)c2cccc([N+](=O)[O-])c2)cc1F', 'Oc1ccc([N+](=O)c2cc(Cl)ccc2Cl)cc1O', 'Oc1ccc([N+](=O)c2ccccc2CO)cc1O', 'Oc1ccc(O[C@@H]2CCC[NH2+]C2)cc1Cl', 'Oc1ccc(O[C@@H]2C(=O)Nc3ccccc32)c(Cl)c1Cl', 'Oc1ccc(O[C@@H]2C[C@@H](c3cccc(COC(F)(F)F)c3)C2)cc1', 'Oc1cc(O[C@@H]2Oc3ccccc3Oc3ccccc32)ccc1N1CCC[NH+]1CCc1ccccc1', 'Oc1cc(O[C@@H]2CCN(c3cccc(Cl)c3)S2)cc2c1OCCO2', 'Oc1cc(O[C@@H](c2nc3ccccc3s2)C[C@H]2CCCO2)cc(O)c1', 'Oc1cccc(F)n1', 'Oc1cccc(NCc2ccc(-c3ccccc3)cc2)c1', 'Oc1cccc(NC2CCN(C(=O)N[C@@H]3CCN(C(=O)c4ccncc4)CC3)C2)c1', 'Oc1cccc(Br)c1O', 'Oc1cccc(Br)c1OCCCc1c(F)cccc1Cl', 'Oc1cccc(Br)c1O/N=C1/C(=O)C2=C(c3ccccc3)CS[C@H]1N2', 'Oc1cccc(Br)c1Oc1ccccc1C#N', 'O=C(/N=C\\NC1=NC=NS1)c1csc(-c2ccc(F)cc2)n1', 'O=C(/N=C\\Nc1cccc(N2CCCC2)c1)c1ccc2c(n1)CCN(Cc1csc(Cl)c1)C2', 'O=C(/N=C\\N1CCN(c2c(C#N)cc(F)cc2F)CC1)c1ccccc1', 'O=C(/N=C/c1ccc(Cl)c2ncccc12)N1CCN(S(=O)(=O)c2ccccc2)CC1', 'O=C(/N=C\\c1occc1OCc1ccc(Cl)cc1)C1CCCC1', 'O=C(/N=C\\c1cc2ccccc2[nH]1)c1cccc(Cl)c1', 'O=C(/N=c1/[nH]c[nH+]c2ccccc12)c1ccc2c(c1)C(=O)N(C1CCCC1)C2', 'O=C(/N=c1/[nH]c(C(F)(F)F)cc(-c2ccncc2)n1)[C@@H]1CCC[NH+](C2CC2)C1', 'O=C(/N=c1/[nH]ccn1CC1CC1)c1csc(-c2c[nH]cn2)n1', 'O=C(/N=c1/[nH]c2ccccc2cc1C(F)(F)F)C(=O)N1CCC[C@@H](c2nnc[nH]2)C1', 'O=C(/N=c1/[nH]c/nc2ccc(Br)cc2c1=O)c1ccc(C2CCCC2)cc1', 'Oc1cc(O[C@@H]2OC(=O)c3ccccc3C2)ccc1OCc1ccccc1', 'Oc1cc(O[C@@H]2Oc3ccccc3OC2)ccc1NCc1ccc(Cl)c(Cl)c1', 'Oc1cc(O[C@@H]2ON=C(c3ccccc3)C[C@H]2C(=O)[O-])ccc1O', 'Oc1cc(O[C@@H]2O[C@@H]3CC[C@@H]2C3)ccc1Oc1ccccc1', 'Oc1cc(O[C@@H]2O[C@H](CO)c3ccccc32)ccc1OCC(F)(F)F', 'O=[N+]([O-])c1cc(Cl)ccc1CCOc1ccccc1F', 'O=[N+]([O-])c1cc(Cl)nc(Nc2ccc(Br)cc2F)c1', 'O=[N+]([O-])c1cc([C@H]2CC(=O)N(c3ccccc3)c3ccccc3C2)ccc1N1CCOCC1', 'O=[N+]([O-])c1cc([C@H]2OCCN2CCCN2CCOCC2)c2cn[nH]c2c1', 'O=[N+]([O-])c1cc([C@H]2[C@@H]3OCC[C@H]3CC[C@H]2O)cc([N+](=O)[O-])c1O', 'O=[N+]([O-])c1cc([C@H]2SCCCN2C(=O)COc2ccc(Cl)cc2)ccc1O', 'O=[N+]([O-])c1cc([C@H]2[C@H]3CCCC(=O)[C@H]3[C@H]3CCCC[C@H]3O2)ccc1C(F)(F)F', 'Oc1ccc([N+](=O)c2cc([N+](=O)[O-])ccc2O)cc1O', 'Oc1ccc([N+](=O)c2cc(C(F)(F)F)ccc2Br)cc1', 'Oc1ccc([N+](=O)c2cc(O)ccc2F)cc1O', 'Oc1ccc([N+](=O)c2cc(F)cc(F)c2)cc1O', 'Oc1ccc([N+](=O)c2cc(Cl)ccc2O)c2ncnn12', 'Oc1ccc([N+](=O)c2cc(Br)ccc2O)cc1', 'Oc1ccc(/C=N\\CCN2CCN(S(=O)(=O)c3ccc4c(c3)OCO4)C[C@H]2c2ccccc2)cc1', 'Oc1ccc(/C=N\\NC2=NC=NC3=C2SCC3)cc1', 'Oc1ccc(/C=N\\c2ccccc2Cl)c(Cl)c1', 'Oc1ccc([C@H]2CC[NH2+]C[C@@H]2C(=O)[O-])cc1F', 'Oc1ccc([C@H]2C(C#N)=C(Nc3ccccc3)NC(=O)N(c3ccc([N+](=O)[O-])cc3)N2)cc1', 'Oc1ccc([C@H]2C=C(c3ccc(F)cc3)Nc3ncnn32)cc1', 'Oc1ccc([C@H]2CN(Cc3csc(Cc4cccc(F)c4)n3)CC2)cc1', 'Oc1ccc([C@H]2C[C@@H]2C(=O)[O-])cc1F', 'Oc1ccc([C@H]2C[NH+]=C(N3CCOCC3)Nc3ncnn32)cc1', 'Cc1snnc1-c1cc2cc([N+](=O)[O-])ccc2o1', 'Cc1snnc1C(=O)N1CCc2cc(O)ccc2[C@H]1c1cccc(O)c1', 'Cc1snnc1S(=O)(=O)Cc1ccc([N+](=O)[O-])cc1', 'C[C@@H](CNC[C@H]1CCCCC[NH2+]1)S(=O)(=O)c1nc(C2CCCCC2)no1', 'C[C@@H](CN1CCCCC1=O)[NH2+]CC(=O)Nc1ccccc1C(F)(F)F', 'ClCc1nonc1CN1C(=O)c2cc(Cl)ccc2S1(=O)=O', 'ClCc1nonc1NC(=O)Cc1csc(N2CCCC2)n1', 'ClCc1nonc1[C@@H]1CCCN(c2ncccn2)C1', 'ClCc1nonc1-c1cc(Br)cc(Br)c1', 'ClCc1ccn(C[C@H](O)c2ccccc2)c1', 'ClCc1ccnn1Cc1ccccc1Oc1ccccc1F', 'ClCc1csc(-c2ccccc2F)n1', 'ClCc1csc(SCc2noc(-c3ccccc3O)n2)n1', 'ClCc1csc(S(=O)(=O)N(Cc2ccccc2)[C@H](CO)c2ccccc2Cl)c1', 'ClCc1csc(Sc2cc(F)ccc2Br)n1', 'ClCc1csc(S[C@@H](CO)c2ccc(F)c(F)c2)n1', 'ClCc1csc(S[C@H]2CCN(Cc3ccccc3)C2)n1', 'ClCc1n[nH]c2ccc(F)cc12', 'ClCc1n[nH]c2nccc(-c3ocnc3)c12', 'ClCc1n[nH]c(C2CC2)n1', 'ClCc1n[nH]c(=O)n1CCOc1ccccc1C(F)F', 'ClCc1n[nH]c(Nc2ccccc2Cl)n1', 'ClCc1n[nH]c([C@@H]2c3ccccc3C[C@]2([NH3+])C(=O)[O-])c1', 'ClCc1n[nH]c(-c2cncc(F)c2)n1', 'ClCc1n[nH]c(S[C@@H]2CCCC[C@H]2Cn2ncn(CC3CC3)c2=O)n1', 'ClC(F)(CCO)N1CCc2sccc2C1', 'ClC(F)(F)C1CCN(C(=O)[C@@H]2CCCOC2)CC1', 'ClCc1ccnnc1', 'ClCc1ccnn1Cc1c[nH]c(=O)nc1N1CCCC1', 'ClCc1cn[nH]c1NC[C@H]1CNc2ccccc2C1', 'ClC(F)/C=C/C=C/C=C/C=C/c1ccc(Cl)c(-c2nc(C3CC3)cs2)c1', 'ClCc1ccn(-c2ccccc2Cl)n1', 'ClCc1ccn(C[C@@H](O)c2cccnc2)c1', 'ClCc1ccn([C@H]2CCO[C@@H]2C(F)(F)F)c1', 'ClCc1cn[nH]c1', 'ClCc1cn[nH]c1C1CCN(c2cccnc2)CC1', 'ClCc1cn[nH]c1O[C@@H]1CCCC[C@@H]1OC1CCCC1', 'ClCc1cn[nH]c1N1CCCSCC1', 'ClCc1cn[nH]c1[C@@H]1CCCN1c1ccc(F)cc1C(F)(F)F', 'ClCc1cn[nH]c1n1cccn1', 'ClCc1cn[nH]c1-c1ccccc1', 'ClCc1cn[nH]c1SCC[S@](=O)Cc1ccccc1', 'ClCc1cn[nH]c1[C@H]1CCCN1c1ccc2c(c1)OCO2', 'ClC(/N=C1/SC2=C(CCCC2)SC1=S)c1ccccc1', 'ClC(/N/N=C\\C=C/c1ccccc1O)C(=O)Nc1ccccc1C(F)(F)F', 'O=C(c1cccnc1)N1CCC(Nc2ccccn2)CC1', 'O=C(c1cccn1-c1ccc(Br)cc1)N1CCC[C@H]1CC(=O)N1CCCC1', 'O=C(c1cccn2cc(Cl)cc12)N1CCC(C(=O)OCc2ccccc2)CC1', 'O=C(c1ccc[nH]c1=O)N1CCN(c2nccc(Br)n2)CC1', 'O=C(c1cccc(-n2cccc2)c1)N1CCC(Oc2cccnc2Cl)CC1', 'O=C(c1ccccn1)Nc1cccc(NC(=O)c2ccco2)c1', 'O=C(c1ccccc1F)N1CCC[C@@H](n2nccc2CS(=O)(=O)C(F)(F)F)C1', 'O=C(c1cccc2ccccc12)N1CCO[C@@H]2CCCC[C@@H]21', 'O=C(c1cc(Br)c[nH]1)c1cc(F)ccc1OCCN1CCCC1=O', 'O=C(c1cc([N+](=O)[O-])ccc1Cl)N1CCN(Cc2cc(C3CC3)cs2)CC1', 'O=C(c1cc(Cl)c2cccnc2n1)N1CC[C@@H](c2ccccn2)C1', 'O=C(c1cn2ccsc2c1)N1CCC(c2[nH+]ccn2CCC(c2ccccc2)c2ccccc2)CC1', 'O=C(c1cn2ccsc2n1)[C@@H]1C[C@H]1c1ccccc1F', 'O=C(COCc1ccccc1)NC[C@H]1CCC[C@@H](O)C1', 'O=C(C1=CN(c2cccc(Oc3ccccc3)c2)CCC1)N1CCc2nc[nH]c2C1', 'O=C(C1=CCCN1c1cccc2cnccc12)Nc1ccc2c(c1)OCO2', 'O=C(C1=C(NC2=NN=C(SCc3ccccc3F)N=N2)[C@@H]2CS(=O)(=O)C=C12)Nc1ccc(Br)cc1', 'O=C(C1=Cc2cc(F)ccc2OC1)N1CCCc2ccsc21', 'O=C(c1ccc(C(F)(F)F)cc1)N1CCC(Oc2ccc(-c3ccccc3)s2)CC1', 'O=C(c1ccc(=O)n(Cc2ccccc2)c1)N1CCC(OC[C@H]2CCCO2)CC1', 'O=C(c1ccc(OCc2ccccc2)cc1)N1CCc2ccccc21', 'O=C(c1ccc(NC(=O)c2sc(N3CCOCC3)nc2C2CC2)cc1)N1CCC[C@@H]1c1ccc(Cl)cc1', 'O=C(c1ccc(F)cc1F)C(=O)Nc1ccc(C(=O)[O-])cc1', 'O=C(c1ccc(-n2cncn2)cc1)N1CCN(c2cc(-c3ccccc3)ncn2)CC1', 'O=C(c1ccc(S(=O)(=O)N2CCOCC2)cc1)N1CCN(c2nc3ccc4ccccc4c3c(COc3ccccc3)nc2=O)CC1', 'O=C(c1ccc(Cl)cc1)N1CCOCC1', 'O=C(c1ccc(Br)cc1)N1CCC(OC(F)F)CC1', 'O=C(c1cnnn1-c1ccc(F)cc1)N1[C@@H](C(=O)[O-])C[C@]2(CCCc3ccccc32)N1Cl', 'O=C(COCc1ccccc1[N+](=O)[O-])Nc1c(F)cccc1F', 'O=C(COCc1nc[nH]n1)NCCc1ccccn1', 'O=C(Cc1cc(F)ccc1F)N1CCC[C@@H]1c1cccc(F)c1F', 'O=C(Cc1ccco1)NC[C@H](CC(F)(F)F)c1ccccc1', 'O=C(Cc1cc2c(cc1Br)OCO2)N1CCN(Cc2ccccc2)CC1', 'O=C(COCc1csc(Br)c1)OCc1ccc(F)cc1', 'O=C(COCc1ccccc1)N/N=C/c1ccc(OCc2ccccc2)cc1', 'O=C(CNC(=O)c1ccc2c(c1)N(C(C)C)CCC2)NCC(=O)[O-]', 'O=C(Cc1nn[nH]c1C(=O)c1ccc2c(c1)CCCC2)c1ccccc1', 'O=C(Cc1nn(-c2ccccc2)c([O-])c1Cl)Nc1cccc([N+](=O)[O-])c1', 'O=C(Cc1nnc(-c2ccc(Br)cc2)o1)N[C@@H](c1cscc1)c1ccccc1', 'O=C(Cc1nnnn1-c1ccc(Br)cc1)Nc1ccc2c(c1)OCCO2', 'O=C(COCCCc1cccc(Cl)c1)N[C@@H]1CCc2c(O)cccc21', 'O=C(COCC(F)(F)F)NOCc1cccs1', 'O=C(COCC1CCN(S(=O)(=O)c2ccccc2)CC1)N1N=C[C@@H](OC2CC2)c2ccccc21', 'O=C(COCCO)Oc1ccccc1F', 'O=C(COCCNc1ccc(Br)cc1)N1CCN(c2cccc[nH+]2)CC1', 'O=C(CNCc1ccc([N+](=O)[O-])cc1)Nc1c(F)cccc1F', 'O=C(CNCc1cccs1)Nc1ccc(Cl)c(Cl)c1', 'O=C(CNCc1noc2ncc(-c3ccccc3)cc12)Nc1ccccc1F', 'O=C(CNc1c(F)cccc1F)Nc1ccc(Oc2ncccn2)cc1', 'O=C(CNc1ccc(Cl)cc1[N+](=O)[O-])Nc1ccc(F)c(F)c1', 'O=C(CNc1cnccn1)N(c1ccc2ccccc2n1)c1ccccc1', 'O=C(Cc1coc(-c2ccccc2)n1)NCc1ccccc1', 'O=C(Cc1conc1-c1cnn(C(C)(C)C)c1)N1CCC(NC(=O)c2cccnc2)CC1', 'O=C(CNS(=O)(=O)c1ccc(F)cc1Cl)NCc1ccccc1', 'O=C(CNC(=O)CN1C(=O)[C@@H]2CC=CC[C@H]2C1)Cc1cc2ncccc2s1', 'O=C(C1CCCC1)N1CCS(=O)(=O)c2ccccc21', 'O=C(C1CCC(Nc2nccs2)CC1)N1CCC(n2ccccc2=O)CC1', 'O=C(C1CCC1)N1CCN(C(=O)c2cnc(CN3CCOCC3)s2)CC1', 'O=C(Cc1cn(-c2ccc(F)cc2F)nn1)NCc1ccccc1', 'O=C(Cc1cnc2ccccc2c1)Nn1ncc(Cl)c1C(F)(F)F', 'O=C(Cc1cn2ccsc2n1)NNC(=O)c1ccc(Cl)s1', 'O=C(Cc1cnoc1)N1CCC(CC[NH+]2CCCCC2)CC1', 'O=C(Cc1cnn(Cc2ccccc2)c1)NC[C@@H](CCO)[NH+](C)[C@@H](C)c1ccccc1', 'O=C(c1c[nH]nc1-c1ccccc1)N1CCSc2ccccc21', 'O=C(c1c[nH]nc1-c1cccc2c1cnn2-c1ccccc1)Nc1cc(F)ccc1F', 'O=C(c1c[nH]nc1-c1cccs1)N1CC[C@@H](Oc2ccccc2)C1', 'O=C(c1c[nH]nc1-c1ccsc1)N1CCN(C(=O)[C@@H]2CCCc3ccccc3C2)CC1', 'O=C(c1c[nH]nc1-c1ccoc1)N1CC[C@@H](c2ccc(F)c(F)c2)C1', 'O=C(c1c[nH]nc1-c1cc(Cl)ccc1Cl)N1CCOCC1', 'O=C(c1c[nH]nc1-c1ccccc1)N1CCN(c2cnccn2)CC1', 'O=C([C@H]1c2ccccc2C(=O)NC12CCCN(c1cccc(Cl)c1)C2=O)Nc1cc(Cl)ccc1Cl', 'O=C([C@H]1c2ccccc2C(=O)N(c2c(O)cccc2Cl)C1=O)N1CCCCN1CC1=CCCCC1', 'O=C([C@H]1c2cc([N+](=O)[O-])ccc2O1)N1CCSC[C@H]1C1OCCO1', 'O=C([C@H]1c2ccccc2CCN1C(=O)C1(c2ccccc2)CCC1)NCc1ccco1', 'O=C([C@H]1[C@H]2C=C[C@@H](C2)[C@H]1C(=O)[O-])C1CCCCC1', 'O=C(COc1cc([N+](=O)[O-])cc(C(=O)[O-])c1)N/N=C/c1cc(Br)ccc1F', 'O=C(COc1cc(C(F)(F)F)ccc1Cl)N(Cc1ccccc1)S(=O)(=O)c1ccccc1', 'O=C(COc1cc(N2CCCCC2)ncc1O)N1CCN(C(=O)c2cc(F)ccc2Br)CC1', 'O=C(COc1cc(F)ccc1N1CCCC1=O)c1ccccc1', 'O=C(COc1cc(Cl)ccc1F)NNC(=O)CCC(=O)c1ccc(F)cc1F', 'O=C(COc1cc(Br)ccc1O)NCC(=O)Nc1ccc2n[nH]ccn2c1', 'O=C([O-])/C=C/c1cn(-c2ccccc2)nn1', 'O=C([O-])/C=C(/C(=O)NCc1noc(-c2ccc(F)c(F)c2)n1)c1ccccc1O', 'O=C([O-])[C@@H]1CC[C@@H](CNC(=O)Nc2ccccc2O)N1', 'O=C(CSC(c1ccccc1)c1ccc(Cl)cc1)Nc1ccccc1C(=O)N1CCCC1', 'O=C(CSC(=S)N1CCc2ccccc21)NCCc1ccccc1', 'O=C(CSC(O)(CO)OCc1ccccc1)NCc1cccs1', 'O=C(CSC(NCC1(C(=O)O)CCCC1)N1CCCC1)N1CCCC1', 'O=C(CSC(F)(F)F)N1CCCSCC1', 'O=C(CSC(/[O-])c1ccc(Cl)c(Cl)c1)NNc1ccccc1Cl', 'O=C(c1cc(F)ccc1F)N(Cc1ccccc1)Cc1cccs1', 'O=C(c1cc(Br)c[nH]1)C1CCCCC1', 'O=C(c1cc(F)c(F)cc1F)N1CCC([NH2+]CC[NH+](C)Cc2ccccc2)CC1', 'O=C(c1cc(F)ccc1F)N1CCC[C@@H]1CSc1ncnc2c1cnn2CC(F)(F)F', 'O=C(c1cc(F)c2c3ccccc3[nH]c(=O)n12)N1CCOCC1', 'O=C([O-])[C@H](CCc1cc(Br)ccc1F)N1C[C@@H]2CC=CC[C@@H]2C1', 'O=C([O-])[C@H](c1ccccc1)n1cnnn1', 'O=C([O-])[C@H](Oc1ccccc1F)C(F)(F)C(F)(F)F', 'O=C([O-])[C@H](NS(=O)(=O)c1ccc(Cl)cc1)c1cscc1C(F)(F)F', 'O=C([O-])[C@H]([C@@H](C(=O)n1cncn1)c1ccccc1)c1cccc(Cl)c1', 'O=C([O-])[C@H]([C@H](O)Cc1ccc(CC(F)F)cc1)N1CCOCC1', 'O=C([O-])[C@H](Br)[C@@H]1CCCO1', 'O=[N+]([O-])c1c(F)ccc(CNCc2nc(-c3ccncc3)no2)c1Cl', 'O=[N+]([O-])c1c(F)cccc1NCCOCc1ccccc1', 'O=[N+]([O-])c1c(F)ccc2c1N(Cc1ccc(Br)cc1)CC2', 'O=[N+]([O-])c1c(F)cc2[nH]ccc2c1Oc1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccn1Cc1cccc(Cl)c1', 'O=[N+]([O-])c1c(Br)c(O)c2c(c1C(N)=O)CC2', 'O=[N+]([O-])c1c(Br)cnc2cccnc12', 'O=[N+]([O-])c1c(Br)cc(F)cc1NC[C@H](O)c1ccc(Cl)cc1', 'O=[N+]([O-])c1c(F)cc2n1CN1CCCC[C@H]1CO2', 'O=C([C@H]1c2ccc(Br)cc2C1)N1CCC(NCc2cccc(F)c2)CC1', 'O=C([C@H]1c2ccccc2CCc2sccc21)N1CCOCC1', 'O=[N+]([O-])c1c([N+](=O)[O-])cc([N+](=O)[O-])c2ccccc12', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(C(=O)N(CCC(F)(F)F)C2CCOCC2)c2ccccc12', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(O)c(=S)n1Cc1cccs1', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(I)cc1N1[C@@H]2CC[C@H]1C[C@@H]2c1ccc(F)cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(F)cc1NCc1ccco1', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(Cl)cc1Br', 'O=[N+]([O-])c1cccc2[nH]ccc12', 'O=[N+]([O-])c1cccc2c1CCN(C[C@H](O)CN1CCc3cc(F)cc(F)c3C1)C2', 'O=[N+]([O-])c1ccc2ncn(CC(=O)N3CCC([C@H]4CCOC4)CC3)c2c1', 'O=[N+]([O-])c1ccc(F)cc1CNC(=O)N1[C@@H](C(=O)[O-])CCN1c1ccccc1', 'OCc1nnc(-c2ccc(Cl)cc2F)[nH]1', 'OCc1nncn1Cc1cccc(F)c1F', 'OCc1nnc2n1CC[C@H]2O', 'OCc1nnsc1N1CCOCC1', 'OCc1nnsc1COc1cc(Cl)ccc1Cl', 'OCc1nnsc1N/N=C/c1cccc(OC2CCCC2)c1', 'OCc1nnsc1-c1ccc2c(c1)OCO2', 'OCc1nnsc1[C@H]1CCCN1C(=O)c1ccccc1', 'OCc1nnsc1Br', 'OCS[C@H](CCc1ccccc1)c1ccc2c(c1)OCO2', 'OCS[C@H]1CCCO[C@]2(CC[NH2+]C[C@H]2c2ccccc2)[C@@H]2OC(O)=C2CC1', 'O=C(C1CCN(c2ccc(F)cc2F)CC1)C1CCC(c2cn[nH]c2)CC1', 'O=C(CNS(=O)(=O)c1cc(Cl)ccc1CN1C(=O)c2ccccc2C1=O)c1ccccc1F', 'O=C(C1CC1)C1=CC[NH+]([C@H]2CCN(c3cc(F)ccc3F)C2)CC1', 'O=C(/C=C\\N=C1\\Nc2ccccc2C1=O)c1ccccc1Cl', 'O=C(/C=C\\NCc1csc([N+](=O)[O-])c1)c1ccccc1', 'O=C(/C=C\\Nc1ccccc1[N+](=O)[O-])c1ccccc1F', 'O=C(/C=C\\N1CCC(OC2CCCCC2)CC1)c1ccccc1F', 'O=C(/C=C1/NC=CCC1)Nc1ccc(Cl)cc1Cc1ccccc1', 'O=C(/C=C1/C[C@H](c2ccncc2)C(=O)Nc2c1CCCC2)N1CCOCC1', 'O=C(/C=C1/c(=C/C=C/C2=CCCCC2)[C@H]1c1cccc(Cl)c1)c1ccc2c(c1)OCO2', 'O=[N+]([O-])c1c([O-])[nH]c(=O)c2ccc(NC(=O)/C=C/N3CCCC3)cc12', 'O=[N+]([O-])c1c([O-])[nH]c2nccnc12', 'O=C(CN(c1ccc(Cl)cc1)S(=O)(=O)c1cccc(Cl)c1)N1CCCC1', 'O=C(CNc1nnc(SCc2ccccc2)o1)Nc1cccc(Cl)c1', 'O=C(c1ccsc1)N(CCO)Cc1cccnc1', 'O=C(c1ccsc1)Nc1ccccc1OC1CCCC1', 'O=C(c1ccsc1)c1ccccc1COC1CCCCC1', 'O=C(c1ccsc1)[C@@H]1CCCN(Cc2ccccc2F)C1', 'O=C(c1c[nH]c2ccccc12)N1CCN(CC2CC[NH+](Cc3ccccc3)CC2)C1', 'O=C(c1c[nH]c2cc([N+](=O)[O-])ccc12)N1CCN(C(=S)NCC2CC2)CC1', 'O=C(c1c[nH]c2cc(C(F)(F)F)cnc12)N1CCC(c2cnc[nH]2)CC1', 'O=C(c1c[nH]c2cc(F)ccc12)N1CCC(O[C@@H]2CCS(=O)(=O)C2)CC1', 'O=C(c1c[nH]c2cc(S(=O)(=O)N3CCOCC3)ccc12)N1CCC[C@H](O)C1', 'O=C(c1c[nH]c2cc(Cl)ccc21)N1CC=C(c2ccccc2)CC1', 'O=C(c1ccsc1)c1cn(-c2ccc(F)cc2)nc1-c1ccccc1[N+](=O)[O-]', 'O=C(c1ccsc1)c1nc2nc(NC(=O)/C=C\\c3ccccc3)noc2c1Br', 'O=C(c1ccco1)[C@H]1CC(c2ccccc2)=NO1', 'O=C(c1cn[nH]c1-c1ccccc1)N1CCc2cc(S(=O)(=O)N3CCCC3)ccc21', 'O=C(c1cn[nH]c1C(=O)c1ccccc1)N1CCOCC1', 'O=C(c1cn[nH]c1N1CCN(c2ccccc2C(=O)N2CCCC2)CC1)N1CCC[C@H]1c1ccccc1', 'O=C(c1cn[nH]c1)N1CCN(c2ccccc2CNc2nc3ccc(Cl)cc3s2)CC1', 'O=C(c1cn2ccsc2c1Cl)c1ccc(Cl)c(Cl)c1', 'O=C(c1cn2ccsc2c1NS(=O)(=O)c1ccccc1)N1CCC[C@@H]1c1cccc(Cl)c1', 'O=C(c1cn2ccsc2c1)N1CCC[C@H]1c1cccc(F)c1', 'O=C(c1ccsc1)[C@@H]1CN(Cc2cccs2)CCO1', 'O=C(c1ccsc1)[C@@H]1SC(N2CC2)c2ccccc2S1(=O)=O', 'O=C(c1ccsc1)[C@@H]1[C@@H]2C[C@@H]2CC[C@@H]1C[C@@H]1CCCO1', 'O=C([C@H](Nc1ccccc1)[NH+]1CCSCC1)c1ccccc1', 'O=C([C@H](Nc1ncccc1Br)C1CC1)Nc1nccs1', 'O=C(/C=C/c1cccc([N-]S(=O)(=O)c2ccc(Cl)cc2)c1)N(Cc1ccccc1)C1CC1', 'O=C(/C=C/c1c(Cl)ccc2ccccc12)N1CCCC[C@@H]1C[C@@H](O)CO', 'O=C(/C=C/c1cccc(Cl)c1)NOCC1(O)CCOCC1', 'O=C(COCc1noc2ccccc12)Nc1ccc(Br)cc1', 'O=C(COCc1n[nH]c(=O)c2ccccc12)N1CCCCCC1', 'O=C(COCc1nc2ccccc2c(=O)[nH]1)Nc1nc(C(F)(F)F)n[nH]1', 'O=C(COCc1nnc2n1CCN(Cc1ccccc1Cl)CC2)N[C@@H](c1ccc(Cl)cc1)C(F)(F)F', 'O=[N+]([O-])c1c(Cl)nsc1C(=O)N1CC[C@@H](Oc2ccc(F)cc2)C1', 'O=[N+]([O-])c1c(Cl)nsc1Oc1ccc(F)cc1F', 'O=[N+]([O-])c1c(Cl)nsc1Nc1cccc(Br)c1', 'O=[N+]([O-])c1c(Cl)nsc1-c1cccc(F)c1', 'O=[N+]([O-])c1c(Cl)nsc1Cl', 'OCc1nc2ccccc2c(=O)n1-c1sc2c(c1-c1ccccc1)CCCC2', 'OCc1nc2ncn(-c3ccc(Br)cc3)c2c(Cl)c1Cl', 'OCc1ncn(-c2ccc(Cl)cc2Cl)n1', 'OCc1ncn[n-]1', 'OCc1ncnc(OC2CCCCCC2)c1N', 'OCc1ncnn1C1CC1', 'O=[N+]([O-])c1c(O[C@@H](CO)c2ccncc2)cccc1[N+](=O)[O-]', 'O=[N+]([O-])c1c(O[C@@H]2CCCC[C@@H]2C(=O)[O-])ccc2cccnc12', 'Oc1cc(-n2cnnn2)ccc1Nc1ncc([N+](=O)[O-])cc1Br', 'Oc1cc(-n2cnnn2)c(NC2CC2)c(Cl)c1', 'Oc1cc(-n2cnc(Cc3ccccc3)n2)ccc1Oc1ccc(Br)cc1', 'Oc1cc(-n2cnc(NNC(=O)C3CCCC3)c2Cl)cc2c1CCC(=O)N2', 'Oc1cc(-n2cnc([C@@H]3CCCN(c4ccccc4OC(F)F)c3)ncn2)ccc1O', 'Oc1cc(-n2cnc(-c3csc(-n4ncn(CCO)c4=O)c3)c2-c2ccccc2)ccc1O', 'Oc1cc(-n2cnc(Sc3nc(C4CC4)ns3)c2[O-])ccc1O', 'Oc1cc(-n2cnc(Cl)n2)ccc1OCCN1CC[NH2+]CC1', 'Oc1cc(-n2cnc(Br)n2)ccc1N1CCOCC1', 'O=C(/C=C\\c1cccnc1)NNc1cccc(NC(=O)c2cccnc2)c1', 'O=C(/C([O-])(c1ccc(F)cc1)c1cnc(Cl)cc1)[C@@H]1SCCS1', 'O=C(c1c[nH]cn1)N(CCNc1ccc(OC(F)(F)F)cc1)C1CC1', 'O=C(c1c[nH]cn1)N1CCCSCC1', 'O=C([C@H](c1ccc(F)cc1)N1CCN(c2ccc(Cl)cc2)CC1)c1ccc2ccccc2c1', 'O=C([C@H](c1ccccc1)N(CC(F)(F)F)C1CCCCC1)C1CCCC1', 'O=C([C@H](c1ccc2c(c1)OCCCO2)N1CCOCC1)N1CCN(Cc2ccccc2)CC1', 'O=C([C@H](c1cccs1)N(c1cccc(Cl)c1)N1CCCC1)c1cccc(F)c1', 'O=C([C@H](c1ccc[nH]1)[NH+]1CCCCC1)N1CCN(c2nc3ccccc3nc2S(=O)(=O)C2CCCC2)CC1', 'O=C(/C=C\\c1c(OC2CCCC2)cc(O)c2c1CSCO2)N1CCCCC1', 'O=C(/C=C\\c1nc(-c2ccc(Cl)s2)cs1)c1ccccc1', 'O=C(/C=C/c1ccc(C(F)(F)F)cc1)OCCCCc1ccccc1', 'O=C(/C=C/c1ccncc1)N/N=C/c1ccc([N+](=O)[O-])cc1', 'O=C(/C=C/c1c(O)oc2cccnc12)COC1CCCCC1', 'O=C(/C=C/c1c(N)n(CCCO)c1=O)c1ccccc1', 'O=C(/C=C/c1c(F)cccc1Cl)Nc1ccc([N+](=O)[O-])cc1', 'O=C(/C=C/c1c(-c2ccccc2Cl)noc1-c1cccnc1)Nc1cccc(C(=O)[O-])c1', 'O=C(/C=C/c1c(Cl)cccc1Cl)NC[C@@H]1CN2CCCC[C@H]2CO1', 'O=C(/C=C\\c1n[nH]c(=O)[nH]1)c1ccc2ccccc2c1', 'O=C(/C=C\\c1nc2sccn2c1Cl)N[C@@H]1CCOC1=O', 'O=C(c1cn[nH]n1)N1CCC(NC(=O)c2ccc(Cl)o2)CC1', 'O=C(c1cn[nH]n1)N1CCC[C@H](c2[nH+]ccn2Cc2cccc(F)c2F)C1', 'O=C(Cc1n[nH]c(=O)[nH]1)NC1(c2ccccc2)CC1', 'O=C(Cc1n[nH]nc1C1CC1)NCC1CC[NH+](CC2(c3ccccc3F)CCCC2)CC1', 'O=C(C1=Nc2sccc2[C@@H]1O)[C@H]1CSC(=S)N1c1ccccc1', 'O=C(C1=NO[C@@H](COc2ccc([N+](=O)[O-])cc2)N1)Nc1ccc(N2C(=O)CCCC2=O)cc1', 'O=C(C1=NN(c2cccc(O)c2)C(=O)N=C1[O-])N1CCO[C@@H](c2ccccc2)C1', 'O=C(C1=N[C@@H](CC(F)F)N1c1ccc(Cl)cc1)N[C@@H]1CCS(=O)(=O)C1', 'O=C(C1=NS(=O)(=O)c2ccccc21)N1CCN(S(=O)(=O)CC2CCC2)CC1', 'Oc1cccc(-c2cc(CNc3ccc(Cl)cc3)nn2-c2ccccc2)c1', 'Oc1cccc(F)n1\n', 'Oc1cccc(Nc2nnc(SCc3nc4cccc(F)c4s3)n2N2CCOCC2)c1', 'Oc1cccc(-c2cc(C3CC3)nc3ccccc23)c1', 'Oc1cccc(-c2cnc3cc(Cl)ccn3c2=O)c1', 'Oc1cccc(-c2csc(-c3ccccc3)n2)c1', 'Oc1cccc(-c2c[nH]nc2[C@H]2CC(c3ccccc3)=NS2(=O)=O)c1', 'OC1=CC(=O)CC1', 'OC1=C(Nc2nc([O-])c(-c3ccccc3)n3c4ccccc4n23)C2(CCC[NH2+]CC2)C1', 'OC1=C2CCCC[C@@H]2C(=O)N1C(=O)c1ccc(Br)s1', 'OC1=CN=C2S[C@@H](COC3=c4ccccc4=[NH+][C@H]3C(=O)N2C2CC1)SCCO2', 'OC1=C[C@@H](C(=O)Nc2ccccc2O)[C@@H]1C', 'OC1=C[C@H](Sc2c[nH]c3cccc([N+](=O)[O-])c23)C([O-])=N1', 'Cc1sn(-c2ccc(F)cc2)c(=O)c1CCC(=O)Nc1ncnc2sccc12', 'Cc1sncc1C(=O)Nc1nc(-c2ncc(-c3ccc(F)cc3)nc2)no1', 'Cc1sn(-c2ccccc2)c(=O)c1S(=O)(=O)N1CCC[C@H]1Cc1ccccc1', 'Cc1nonc1CCC(=O)NCc1cc(-c2ccc(C#N)cc2)on1', 'Cc1noc(CCc2c(C#N)c([O-])nn2-c2cccc(C)c2)c1C', 'Cc1noc2nc(S[C@@H](C)C(=O)c3ccccc3)sc12', 'CNS(=O)(=O)c1csc(C(=O)N[C@@H](C)C(=O)N2CCCCCC2)c1', 'Clc1ccc(-c2ccnc3ncccc23)cc1F', 'Clc1ccc(-c2ccoc2C2CC(CO)C2)cn1', 'Clc1ccc(-c2noc(-c3cccc(C(F)(F)F)c3)n2)c(Cl)c1', 'Clc1ccc(-c2noc(-c3cc(F)ccc3Br)n2)cc1', 'Clc1ccc(-c2nncn2C2CC2)cc1', 'Clc1ccc(-c2n[nH]cc2-c2ccc(NC[C@H](O)C[NH+]3CCCC3)cc2)c(F)c1', 'Clc1ccc(Br)cc1OC[C@H]1CC=CCC1', 'Clc1ccccc1OC1CCN(c2ncccn2)CC1', 'Clc1ccccc1O/C=C(O[C@H]1CCSCC1)c1ccccc1', 'Clc1ccccc1Oc1ccc(COc2ccc3c(c2)OCCCO3)cc1', 'Clc1ccc(Br)c(Cl)c1', 'Clc1ccc(Br)cc1CN(Cc1ccc(Cl)cc1Cl)C[C@@H](O)C1CC1', 'Clc1ccc(Br)c(C[NH+]2CC[C@@H]3CCCC[C@@H]32)c1', 'Clc1ccc(Br)c(OCc2nnc3ccccn23)c1', 'Clc1ccc(Br)c(N2CCN([C@@H]3CCCC[NH+]3Cc3ccccc3F)CC2)c1', 'Clc1ccc(Br)c(F)c1-c1ccccc1', 'Clc1ccc(Br)c([C@@H]2CCCN(S(=O)(=O)c3cc(F)cc(F)c3)C2)c1', 'Clc1ccc(Br)c(-c2n[nH]c([C@@H]3CCCN(/C=C/CCC[NH+]4CCCC4)C3)n2)c1', 'Clc1ccc(Br)c(S(=O)(=O)N[C@@H]2CCCc3ccccc32)c1', 'Clc1ccc(Br)c(Cl)c1C1CC[NH+](Cc2ccccc2)CC1', 'Clc1ccc(Br)c([C@H]2[C@H]3CCCC(=O)N3C3CC[NH+](CCN4CCCCC4C2)C3)c1', 'Clc1ccc(Br)c(/C=N/c2c[nH]c3ccccc23)c1', 'COc1nn[nH]c1/C=C/C(=O)N(CC#N)C1CC1', 'O=[N+]([O-])c1c(Br)cnc2ccc(F)cc12', 'O=[N+]([O-])c1c(Br)cc(Cl)cc1C(=O)N1CCO[C@@H](c2ccccc2)C1', 'O=[N+]([O-])c1c(Br)cccc1N1CCOCC1', 'O=[N+]([O-])c1c(Br)c(O)nc2ncccc12', 'O=[N+]([O-])c1c(Br)c(I)c2c(n1)c1cc(-c3ccccc3)ccc12', 'O=[N+]([O-])c1c(Br)c(F)cc(OCC(=O)N2CCC[C@@H](Cc3ccccc3)C2)c1', 'O=[N+]([O-])c1c(Br)c([C@@H]2CCCCO2)cs1', 'O=[N+]([O-])c1c(Br)c(-c2cccc(O)c2)nc2[nH]ccc12', 'O=[N+]([O-])c1c(Br)c(Br)cc([O-])c1[N+](=O)[O-]', 'O=C([C@H](Sc1ccccc1F)N1CCC(c2ccccc2)(C(=O)[O-])CC1)n1cccc1', 'O=C([C@H](Sc1n[nH]c2ccccc12)c1ccccc1)N(Cc1ccc(Cl)cc1)C1CC1', 'O=[N+]([O-])c1c(-c2ccsc2)ncc2ccc(OC(F)F)cc12', 'O=[N+]([O-])c1c(F)cccc1Nc1ccc(Cl)cc1[N+](=O)[O-]', 'O=[N+]([O-])c1c(-c2c[nH]nn2)cnnc1NCc1ccc(OC(F)(F)F)cc1Cl', 'O=[N+]([O-])c1c(F)ccc2c1N=C(c1ccccc1)OC2=O', 'O=[N+]([O-])c1c(F)ccc2ncn([C@@H]3CC(=O)Oc4ccccc4N3)c12', 'ClC[C@H](c1c[nH]c2ccccc12)N1c2ccccc2C[C@@H]1Cc1cccs1', 'ClC[C@H](Br)[C@@H]1CCCN(S(=O)(=O)c2cccs2)C1', 'Oc1cccc(F)n1\n\n', 'Oc1cccc(N2CCCN(Cc3ccnc4c3cnn4-c3ccccc3)CC2)c1', 'NS(=O)(=O)/C(=C\\c1ccc(Cl)cc1)c1[nH]c2ccccc2c1Oc1ccc2c(c1)OCO2', 'ClC[C@H]1CCCN(Cc2csc(-c3ccccc3)n2)C1', 'ClC[C@H]1COc2ccc(F)cc21', 'ClC[C@H]1CN(c2ccccc2)CC1', 'ClC[C@H]1C[C@H]1c1cccc(F)c1Cl', 'ClC[C@H]1C[NH+](Cc2ccccc2)CC1', 'COc1nc(NC[C@H]2CCCO2)ccc1S(=O)(=O)NC1CCC(C)CC1', 'COc1nc(N(C2CC2)[C@@H](C)C2CC2)ccc1-c1ccco1', 'COc1nc(N)cc(C(=O)N2CCC[C@@H](CO)C2)c1N', 'COc1nc(Nc2nc(C3CC3)n(-c3ccccc3C)n2)cc2c1OCO2', 'COc1nc(N2CCCCC2)nc2c1cnn2Cc1ccccc1Cl', 'COc1nc(N[C@@H](COC(C)C)Cc2ccccn2)ccc1C', 'COc1nn[nH]c1/C=C/C(=O)Nc1ccc(N2CCCCCC2)cc1', 'COc1nn[nH]c1/C(C)CCc1cccc(F)c1', 'Cc1sncc1-c1c(C)nc2[nH]c(C(C)C)nc2c1[O-]', 'Cc1sncc1C(=O)O[C@@H]1[C@@H](O)[C@H](C)OC[C@@H]1OCc1ccccc1', 'Cc1sncc1SCC(=O)N1CCN(c2cccc[nH+]2)CC1', 'Cc1sncc1[C@@H](C)Oc1cnc(Oc2cccc(Cl)c2)c(C#N)c1', 'Clc1nncn1-c1cccc(Cl)c1', 'Clc1nncn1-n1cnc(COCCN2CCOCC2)c1C1CC1', 'Clc1nncn1c1ccc(Cl)c(Cl)c1', 'Clc1nncn1/C(CO)C[C@H]1CCC[C@@H]1O', 'Clc1nncn1/n1/nc(CSc2ccc(F)cc2)n2ccnc21', 'Clc1nnc(S(=O)(=O)NC[C@@H]2CCO[C@@H]2CO)cc1F', 'Clc1nnc(Sc2cccc(I)c2)nc1N1CCCC1', 'Clc1nnc(S[C@@H]2CCC[C@@H](O/N=C/c3ccccc3)C2)n1C1CC1', 'Clc1nnc(S[C@H]2CC(=O)N(c3ccc(F)cc3)C2)n1Cc1ccco1', 'O=C(c1cnn(CCO)c1)[NH+]1CCC[C@@H]1CC#N', 'O=C(c1cnn(C(=O)N[C@@H]2CS(=O)(=O)C[C@@H]2C(=O)[O-])c1)c1ccccc1', 'O=C(c1cnn(Cc2ccccc2C#N)c1)N1CCCCC1', 'O=C(c1cnn(C2CCCC2)c1)N1CC[C@H](C(=O)[O-])c2ccccc21', 'O=C(c1cnn(C[C@@H]2CCCO2)c1)N1CCC(c2nc3c(F)cccc3s2)CC1', 'O=C(c1cnn(C[C@H]2CCCO2)c1)N1CCCCC1', 'O[C@@H](c1csc(C2CCCCC2)n1)[C@H]1CCOC(O)(CO)C1', 'O[C@@H](c1csnc1)[C@H]1CC(=O)N(c2cc(Cl)ccc2C#N)C1', 'ClC[C@H]([NH2+]Cc1ccccc1)[C@@H]1CCCC[C@@H]1O', 'ClC[C@H]([NH2+][C@@H](C#N)c1ccccc1)c1csc(Cl)c1', 'Oc1cc(Br)cs1', 'O=[N+]([O-])c1c(-c2cccs2)nc(-c2ccco2)n1Cc1cccs1', 'O=[N+]([O-])c1c(-c2ccco2)sc2ccccc12', 'O=[N+]([O-])c1c(-c2ccc(Br)cc2)n[nH]c1N[C@H]1CCC[C@@H](C(F)(F)F)C1', 'Oc1cccc2cc(CC[NH2+]Cc3ccncc3)ncc12', 'Oc1cccc2cc(C(=O)OCCN3CCOCC3)cnc12', 'Oc1cccc2cc(COc3cccc(C[N+](=O)[O-])c3)[nH]c12', 'Oc1cccc2cc(CN3CCC[C@@H]3C(=O)c3cccc(C(F)(F)F)c3)oc12', 'Oc1cccc(F)n1\n\n\n', 'Oc1cccc(OCCc2cccnc2)c1O', 'Oc1cccc(OC(F)F)c1NC1=NN=C(c2ccccc2)C1', 'Oc1cccc(OCc2nc(C3CCCCCC3)[nH]n2)c1', 'Oc1cccc(OC2CCN(C/C=N\\CC[NH+]3CCC(Cc4ccccc4F)CC3)C2)c1', 'Oc1cccc(OC[C@@H]2CO2)c1', 'Oc1cccc(OC[C@H]2CCCO2)c1', 'O=C([C@H](Cl)Br)c1ccc(Cl)cc1', 'O=C([C@H]1c2cc([N+](=O)[O-])ccc2Sc2ccccc21)[NH+]1CCN(c2cccc[nH+]2)CC1', 'O=C([C@H]1c2cc(C(F)(F)F)ccc2NC1=O)N1CSC[C@H]1[C@H]1CCCO1', 'O=C([C@H]1c2cc(O)ccc2Oc2ccc(F)cc21)NC1CCCC1', 'O=C([C@H]1c2cc(F)ccc2OC1)N1CCN(c2ccc(C(F)(F)F)cn2)CC1', 'O=C([C@H]1c2cc(S(=O)(=O)[O-])ccc2N=C1N1CCOCC1)NNc1ccccc1C(F)(F)F', 'N#C[C@H]1C(N)=N[C@H]([C@H]2CC[NH2+]C2)c2cc(Br)ccc21', 'N#C[C@H]1NC(=O)C[C@@H](CCl)N1c1ccccc1Cl', 'Oc1cc(Br)cs1', 'Oc1cc(Br)c2sccc2c1OCc1ccc(O)cc1', 'Oc1cc(Br)c2[nH]nc(C(=O)/C=C/c3nnnn3-c3ccccc3)c2n1', 'Oc1cc(Br)c2c(c1)nc(C[C@@H](CO)O)n2CC1CCN(c2ccccc2)CC1', 'Oc1cc(Br)ccc1[C@H](O)c1cc(Cl)ccc1Cl', 'Oc1cc(Br)cs1\n', 'Oc1cc(O)cc(-c2ncccc2OCc2ccccc2)c1', 'Oc1cc(O)ccc1CN1CCN(S(=O)(=O)c2cccc(Cl)c2)CC1', 'Oc1cc(O)cc2c1C[C@H](CNCc1n[nH]c(C3CC3)n1)C2', 'Oc1cc(-n2nc3sc4ccccc4nc32)cs1', 'Oc1cc(-n2nc3c(c2)CCCN3C(=O)c2ccc(Cl)cc2)cc(Cl)c1', 'Oc1cc(Br)cc2ccccc12', 'Oc1cc(Br)cc2c(Cl)cc(SCc3ccoc3)nc12', 'Oc1cc(Br)cc2c1[C@H](N1CC[NH2+]CC1)OCO2', 'Oc1cc(Br)cc2cn[nH]c12', 'Oc1cc(F)ncc1CNC(=O)CSc1ncnc2c1CCCC2', 'Oc1cc(F)ncc1CN(Cc1ccco1)CC(F)(F)F', 'Oc1cc(F)ncc1N[C@H]1CCCn2ccnc21', 'Oc1cc(F)ncc1-c1cccnc1NCc1ccccc1F', 'Oc1cc(F)ncc1S(=O)(=O)c1ccc(F)cc1', 'Oc1cc(F)ncc1[C@H](O)c1cccc(F)c1', 'Oc1cc(O)c2scc(Br)c2c1', 'Oc1cc(O)c2[nH]cnc2c1/N=N/c1ccccc1', 'Oc1cc(O)c2nc(NCc3cccnc3)[nH]c(=O)c2c1', 'O=[N+]([O-])c1cccc2[nH]ccc12', 'O=[N+]([O-])c1cccc2[nH]cnc12', 'O=[N+]([O-])c1ccc2c(c1)CCN(Cc1ccc([O-])cn1)CC2', 'O=[N+]([O-])c1ccc2ccc(Br)cc2c1', 'O=[N+]([O-])c1c([O-])sc2scnc12', 'CC[C@@H]1C(=O)NC[C@@H](C(=O)OC)C1', 'CC[C@@H]1O[C@H](C(=O)N2CCc3c([nH]c4ccccc34)N2C)C(=O)N1', 'CC[C@@H]1c2ccncc2CN1C(=O)c1ccc(NC(=O)C(C)(C)C)cc1', 'Clc1ccccc1CCNC1=NC=CCN1', 'Clc1ccccc1C(F)(F)F', 'Clc1ccccc1Cc1ccc(O[C@@H]2CC[NH2+]C2)c(Br)c1', 'Clc1ccccc1C1CCN(CCSc2ccccc2)CC1', 'Clc1ccccc1COC1CCCCC1', 'Clc1ccccc1CNc1ccc(-n2cccn2)nc1', 'Clc1ccccc1CI', 'Clc1ccccc1Cn1cc[nH+]cc1=S', 'Clc1ccccc1CS(=O)(=O)CC(=O)Nc1ccc(Br)cc1', 'Clc1ccccc1C[NH+]1CCN([C@@H]2CCO[C@]3(CCCO3)C2)CC1', 'Clc1cc(Br)cnc1OCc1cc(F)ccc1Br', 'Clc1cc(Br)cn1CCNC(=O)C1CCN(S(=O)(=O)c2c(F)c(F)c(F)c(F)c2F)CC1', 'Clc1cc(Br)c([N+](=O)[O-])cc1N1CCC[C@H](CO)C1', 'Clc1cc(Br)c(CN2CCCC[C@H]2c2cnco2)cc1Cl', 'Clc1cc(Br)c(Nc2nc([O-])c3ccccc3n2)c(O)c1', 'Clc1cc(Br)c(F)c(/N=C2/S[C@@H]3CSC[C@@H]23)c1', 'Clc1cc(Br)c(-c2ccccc2Cl)cc1Cl', 'Clc1cc(Br)c(S(=O)(=O)N2CCCC2)cn1', 'Clc1cc(Br)c([O-])c(=O)n1/N=c1s/c(=C\\c2ccccc2)n1Cc1ccccc1', 'Clc1cc(Br)c([C@H](O)c2ccccc2)cc1F', 'Clc1cc(Br)c(Br)c(CCNc2ncnc3c2cnn3Cc2ccccc2)c1', 'Clc1cc(Br)c(/C=C/C(=O)[O-])cc1O', 'Clc1cc(Cl)nc(CSc2cc(F)ccc2Cl)c1', 'Clc1cc(Cl)nnc1CSc1ncc[nH]1', 'Clc1ccsc1-c1ccoc1C[NH+]1CCCC1', 'Clc1nncn1c1cnn(C2CCC(Oc3ccc(Cl)cc3)CC2)c1', 'Clc1nncn1[C@@H](CO)c1ncccc1F', 'Clc1nncn1[C@@H]1CCCC[C@H]1C(=O)[O-]', 'Clc1cc(/N=C/c2ccc(-c3ccco3)cc2)cs1', 'Clc1cc(/N=C\\c2ccc(OCc3ccccc3)cc2)nc(Cl)c1', 'Clc1cc(/N=C\\C=C/C[NH+]2CCCC2)ccc1Cl', 'Clc1cc(/N=C\\N2CCCCCCC2)ccc1[N+](=O)[O-]', 'Clc1cc(/N=C\\c2ccco2)c2ccccc2c1O', 'ClC(=S)Oc1ccc(Cl)cc1Br', 'ClC(=S)O[C@@H](c1ccc(Cl)cc1)c1ccccc1F', 'ClC(=S)O[C@H]1CCCC[C@@H]1OCc1ccccn1', 'ClC(=S)O/C=C1\\C(/C=N\\C(=O)[O-])C(=S)NC(=O)N1', 'O=[N+]([O-])c1ccc(-c2n[nH]cc2CC(=O)N2CCCC2)cc1', 'O=[N+]([O-])c1ccc(-n2cccc2)cc1O', 'O=[N+]([O-])c1cccc2c1C[C@H]([NH2+]Cc1ccccc1Cl)C2', 'O=[N+]([O-])c1cccc2c1OCCCN2[C@H]1CCS(=O)(=O)C1', 'O=[N+]([O-])c1cccc2c1N[C@@H](C(F)(F)F)[C@H]1CC=C[C@@H]21', 'O=[N+]([O-])c1cccc2c1[C@@H]1CCC[C@@H]2[C@@H]2C[C@H]([n+]3ccco3)CCC[C@@]21O', 'O=[N+]([O-])c1cccc2c1nc([O-])c1ccccc12', 'O=[N+]([O-])c1cccc2c1[C@H](CO)C2', 'O=[N+]([O-])c1cccc2c1/C(=N/CCCOc1ccccc1)CCCC2', 'O=[N+]([O-])c1cccc2c1[nH]c(=S)n2CC1CCOCC1', 'COC(CC)(NC(=O)c1ccc(C)c(C)c1)C(=O)OC', 'COC(CNC(=O)c1cc2ccccc2oc2ccccc12)c1ccccc1', 'COC(C)(C)CN(C)C(=O)c1ccc[nH+]c1NC(=O)CC1CCC1', 'Clc1nnn(-c2ccccc2)c(=O)c1Br', 'Clc1nnn(-c2ccc(F)cc2)c1CN1CCCCCC1=O', 'Clc1nnn(-c2ccccc2)c1Cn1ccnc1-c1cnccn1', 'Clc1nncc(-c2ccc(F)cc2)n1', 'Clc1nncc(-n2cccc2)c1/C=C/C(=O)N1CCc2ccccc21', 'Clc1nncc(C2CC2)n1', 'Clc1nncc(COc2ccccc2CN2CCOCC2)n1', 'Clc1nncc(CN2CCC[C@@](O)(CCn3cnc4ccccc4c3=O)C2)c1', 'Clc1nncc(C[C@@H]2CCCCc3ccc(F)cc23)c1C1CC1', 'Clc1nncc(CSc2nccnc2C2CC2)n1', 'Clc1nncc(C[NH+]2CCC[C@@H]2c2ncc[nH]2)n1', 'CN=Nc1nc(CNCc2ccccc2O[C@@H]2CCC[C@@H]2OCC(F)F)n[nH]1', 'CN=N[C@H]1CC(c2ccc(Cl)cc2)=[NH+]/C1=N\\OC(C)=O', 'CN=N[C@]1(C(=O)OC)CCc2ccccc21', 'NS(=O)(=O)Nc1ccc([N+](=O)[O-])cc1Cl', 'NS(=O)(=O)N1CCC(CNC(=O)N2CC[C@@H](C(=O)Nc3ccccc3)C2)C1', 'NS(=O)(=O)Nc1cc(Cl)ccc1C(=O)N(CCn1cccn1)C1CC1', 'NS(=O)(=O)Nc1ccc([N+](=O)[O-])c(NCC[NH+]2CCCCCCC2)c1', 'Oc1cc(COC(=O)C2CCN(c3ccc(Cl)cc3)CC2)ccc1Oc1ccccc1', 'Oc1cc(COCc2cccc(Br)c2)c2c(cc3ccccc32)c2ccccc12', 'Oc1cc(COCOC2[C@@H](N3CCOCC3)CCC2)ccn1', 'Oc1cc(COC[C@@H]2CC[NH+](Cc3ccccc3)C2)ccc1OCc1ccccc1', 'Oc1cc(COC[C@H]2CCCCOC2)cc([N+](=O)[O-])c1Cl', 'Oc1cccc2cc(O)ccc12', 'Oc1cccc2cc(OC(F)F)oc12', 'Oc1cccc2cc(OCc3ccc(F)cc3)cnc12', 'Oc1cccc2cc(OC3CCC[C@]23CO)c[nH]1', 'Oc1cccc2cc(OC[C@@H](O)[C@@H](c3ccccc3)C(F)(F)F)cnc12', 'Oc1cccc2cc(OC[C@H]3CCCO3)ccc12', 'Oc1ccccc1/N=C/c1cccc(OCC(F)F)c1O', 'Oc1ccccc1/N=C(\\Nc1c(F)cc(Cl)cc1)c1ccc(Cl)cc1', 'Oc1ccccc1/N=N/c1ccc(C(F)(F)F)cn1', 'Oc1ccccc1/N=c1\\scc(N2CCOCC2)n1-c1ccccc1', 'Oc1ccccc1Oc1ccc2c(=O)nc(NCCO)oc2c1', 'Oc1ccccc1OCCN(C[C@@H]1CCCO1)c1sccc1C1CC1', 'Oc1ccccc1OC(=O)CN1CCc2cc(F)ccc2C1', 'Oc1ccccc1OC1CN(C(=O)c2cc(C3CC[NH+](C)C/C=C[C@H]3CO)[nH]n2)CC1', 'Oc1ccccc1OC[C@@H]1Cc2ccccc2O1', 'Oc1ccccc1OC[C@H](O)COc1ccc2ccc(=O)oc2c1', 'Oc1ccc([N+](=O)[O-])c(Cl)c1OC[C@H]1CCCO1', 'Oc1ccc(F)c(SCC[NH2+]Cc2ccc(Cl)cc2)c1', 'Oc1ccc(F)c(S(=O)(=O)c2cccc(F)c2F)c1', 'Oc1ccc(F)c(Sc2n[nH]cc2C(=O)NCCc2cccc(O)c2)c1', 'Oc1ccc(Oc2nc(COc3ccccc3)no2)cc1', 'Oc1ccc(Oc2nc3ccsc3c(=O)o2)cc1Cl', 'Oc1ccc(Oc2ncnc3sccc23)c2ccccc12', 'Oc1ccc(Oc2nc[nH]n2)c2ncccc12', 'Oc1ccc([N+](=O)CN(C(=O)c2cccc([N+](=O)[O-])c2)c2ccccc2)cc1', 'Oc1ccc([N+](=O)CNc2ccccc2OCC2CCCCC2)cc1', 'Oc1ccc([N+](=O)CN2C(=O)COc3ccccc3[C@@H]2Nc2ccccc2)cc1', 'Oc1cc(OC(C(=O)[O-])c2ccc(C(C)CO)cc2)cs1', 'Oc1cc(OC(F)F)ccc1-c1ccccc1', 'Oc1ccccc1/C(Cl)Oc1cccc(C(F)(F)F)c1F', 'Oc1ccccc1/C([O-])/N=C/c1ccccc1', 'Oc1cccc(F)n1\n\n\n\n', 'Oc1cccc(S[C@@H](C(=O)N2CCC(c3ccc(C(F)(F)F)cc3)CC2)c2ccccc2)c1', 'Oc1cccc(S[C@@H]2CC(c3ccccc3)C(=O)N2c2ccc(C(C)=O)cc2)c1C(F)(F)F', 'Oc1ccccc1-c1ccc(=O)n(Cc2nc(-c3cccc(F)c3)no2)c1', 'Oc1ccccc1-c1ccc(Cl)cc1', 'Oc1ccccc1-c1noc([C@@H]2CCO[C@]3(CCSC3)C2)n1', 'Oc1ccc(O[C@H]2CCOC2)cc1-n1cccn1', 'Oc1ccc(F)c(Cl)c1Cl', 'Oc1ccc(F)c(OCc2ccccc2)c1', 'Oc1ccc(F)c(O)c1', 'Oc1ccc(F)c(Oc2ccccc2C(=O)[O-])c1', 'Oc1ccc(F)c(ON/C=[N+]([O-])c2ccc(Cl)cc2)c1', 'Oc1ccc(F)c(O[C@@H]2CCOC3(CCCC3)C2)c1', 'Oc1ccc(F)c(OS(=O)(=O)c2ccc(I)cc2)c1', 'Oc1ccc(F)c(O[C@H]2CCN(Cc3ccccc3)C2=O)c1', 'Oc1ccc(O[C@H]2C=C[C@H](c3ccccc3)C2)cc1Cl', 'Oc1ccc(O[C@H]2CS(=O)(=O)C[C@@H]2C[NH2+]C2CC2)cc1F', 'Oc1cc(F)ncc1SCC(=O)Nc1ccc2c(c1)OCCO2', 'Oc1cc(F)ncc1S(=O)(=O)N[C@@H]1CCN(c2ccc(F)cc2)C1', 'Oc1cc(F)ncc1Sc1ncnc2ncnc(Nc3ccccc3)c12', 'Oc1cc(F)ncc1S[C@@H](CO)Cc1c[nH]nc1C1CC1', 'Oc1cc(F)ncc1S[C@H](C(=O)[O-])c1ccccc1', 'O=C(CCNC(=O)N1CCOCC1)N1C[C@@H]2CCCN[C@@H]2C1', 'O=C(CCNc1ccccc1N1CCOCC1)NCc1cccs1', 'O=C(CCN1CCOCC1)N(c1ccccc1F)[C@@H]1CCS(=O)(=O)C1', 'O=C(CCN[C@@H]1COc2ccccc21)Nc1ccc2c3c(cccc13)CS2', 'O=C(CCNS(=O)(=O)c1ccccc1)Nc1ccccc1', 'Oc1cccc2ncn([C@@H](Cc3ccccc3)c3ccccc3)c12', 'Oc1cccc2ncoc12', 'Oc1cccc2nc(NC[C@@H](O)Cc3ccccn3)cnc12', 'Oc1cccc2nc(SCCSc3nc4ccccc4cc3n1)CCC2', 'Oc1cccc2nc([C@H](c3ccccc3)N3CC[NH+](C[C@H]4CCCO4)CC3)ncc12', 'Nc1nncn1CCCc1nc(-c2ccccc2F)cs1', 'Nc1nncn2cnnc12', 'Nc1nc(C[C@@H]([NH2+]Cc2cnco2)c2ccc(F)cc2Cl)ccn1', 'Nc1nc(NCC2CCC2)c2c(C(N)=O)cccc2n1', 'Nc1nc(-c2cccs2)nc2scnc12', 'Nc1nc(SCc2cccnc2)cs1', 'Nc1nc([C@H]2CCCc3cccnc3N2)no1', 'O=C(c1c[nH]c2ncn(CC(=O)NCCc3ccccc3)c12)N1CCOCC1', 'O=C(c1c[nH]c2ncnc(NC3CC3)c12)N1CCOCC1', 'O=C(c1c[nH]nc1Nc1nsnc1C(F)(F)F)N1CCCN(C(=O)C(F)(F)F)CC1', 'ClCc1ncn(-c2ccccc2)n1', 'ClCc1ncnc(C2CC2)n1', 'ClCc1ncn2cccc(-c3ccncc3)c12', 'ClCc1ncnn1-c1ncnc2c1CCCC2', 'ClCc1ncn[nH]1', 'ClCc1noc(-c2cnc(-c3ccc(Cl)cc3)nc2)n1', 'ClCc1noc2c1C[C@@H](CO)C2', 'ClCc1nonc1NCCC[NH+]1C[C@H](C)CC[C@H]1Cc1ccccc1', 'ClCc1nonc1Nc1ccc2c(c1)CCCC2', 'ClCc1nonc1N[C@@H]1CC[C@H]([NH3+])C1', 'ClCc1nonc1N/N=C/c1ccc(-c2ccccc2)cc1', 'O[C@@H](Sc1ccc(F)cc1Cl)Cn1ccnc1', 'O[C@@H](Sc1nnc(-c2ccc(F)cc2)o1)c1cc(Cl)ccc1Cl', 'O[C@@H](Sc1n[nH]c2c1CCCC2)c1cccc(OC2CC2)c1', 'O[C@@H](Sc1nnc(C2CC2)n1C1CC1)C(=O)Nc1ccccc1', 'O[C@@H](S[C@@H](F)c1cccnc1)c1ccccc1F', 'O[C@@H](S[C@@H](Cc1ccccc1)c1ccccc1)c1ccco1', 'O[C@@H](S[C@@H](N)c1cccc([N+](=O)[O-])c1)S(=O)(=O)c1ccccc1', 'O[C@@H](S[C@@H](c1ccccc1)c1ccncc1)C1CC1', 'ClC(F)(F)OCCCNc1cc(Cl)ccn1', 'ClC(F)(CCC(F)F)NCc1cc(Cl)cs1', 'ClC(F)(C(F)F)Nc1ccccc1C(F)(F)F', 'ClC(F)(Cc1ccco1)C(=O)c1ccc(Br)s1', 'ClC(F)(CO)[NH2+]Cc1ccc(Cl)cc1', 'ClC(F)(C[C@@H]1CCO[C@]2(CCSC2)C1)C(F)(F)F', 'ClC(F)(CBr)c1c[nH]cn1', 'O=C(c1cccs1)c1csc(NCc2cccc(Cl)c2)n1', 'O=C(c1ccc[nH]c1=O)N(CCOc1ccccc1F)[C@@H]1CCS(=O)(=O)C1', 'O=C(c1cc([N+](=O)[O-])ccc1-n1cccn1)N1CC1(C#N)C1CC1', 'O=C(C[NH+]1CC[C@@H]2CCCC[C@@H]21)NC[C@H]1-c2ccccc2CN1C(=O)c1ccc(F)cc1', 'O=C(C[NH+]1CCOCC1)Nc1cccc(Cl)c1', 'O=C(C[NH+]1CCO[C@H](c2ccccc2)C1)Nc1ccc(Cl)cc1', 'O=C(C[NH+]1CCOc2ncccc2C1)Nc1ccc(Cl)c(F)c1', 'O=C(C[NH+]1CCO[C@]2(CCCN(c3ncccn3)C2)C1)NC1CCCCC1', 'NS(=O)(=O)/N=C/CCc1ccc2c(c1)OCO2', 'NS(=O)(=O)/N=C/C(=O)Nc1cccc(Br)c1', 'NS(=O)(=O)/N=C/Cc1ccccc1Br', 'NS(=O)(=O)/N=C/C1CCCC1', 'NS(=O)(=O)/N=C/C=C/c1ccc(Cl)cc1OCc1ccccc1', 'NS(=O)(=O)/N=C/C[C@@H]1CCC[NH2+]1', 'O[C@@H](c1cc([N+](=O)[O-])ccc1F)[C@H]1C[C@@H]1c1cc(Cl)ccc1Br', 'O[C@@H](c1cc(OC(F)F)cc(Br)c1)[C@@]1([NH+]2CCCCC2)CCOCC1', 'O[C@@H](c1cc(I)ccc1F)[C@@H]1CN(C(F)(F)F)CCO1', 'O[C@@H](c1cc(F)cc(F)c1)c1nc(C2CC2)ns1', 'O[C@@H](c1cc(Cl)ccc1O)c1c[nH]nc1C1CCCCC1', 'O[C@@H](c1cc(Br)ccc1F)c1cccc(F)c1', 'C[C@@H]([NH2+]CC1([NH+]2CCOCC2)CCCCC1)c1nc2ccc(-c3ccccc3)cc2[nH]1', 'C[C@@H]([NH2+]C1CCC(N2CCOCC2)CC1)c1ncnc2c1CCCCC2', 'C[C@@H]([NH2+]C[C@@H]1CCCC[C@H]1CCOc1ccccc1)c1ccc(F)c(F)c1', 'C[C@@H]([NH2+]C[C@H](O)C1CCOCC1)c1cccc(OCC#N)c1', 'Nc1[nH]ncc1N(C(=O)c1ccc(Cl)cc1N(C)Oc1ccccc1Cl)S(C)(=O)=O', 'Nc1[nH]ncc1Nc1ccc(C(=O)Nc2ccccc2F)cc1Cl', 'Nc1[nH]ncc1N1CCN(C(=O)c2c(Cl)cccc2Cl)CC1', 'Nc1[nH]ncc1N[C@@H]1CCN(c2ccccc2)C1=O', 'Nc1[nH]ncc1NS(=O)(=O)CC1CCCC1', 'Nc1[nH]ncc1N[C@H]1CCN=C1Cc1nnc2n1CCC2', 'NS(=O)(=O)N[C@H]1CC[NH+]([C@@H]2CCN(c3cccnc3)C2)C1', 'NS(=O)(=O)N[C@H]1c2ccccc2C(=O)N1CC(N)=O', 'N=Cc1nonc1Cc1ccccc1F', 'N=Cc1cn[nH]c1SCC(=O)NCc1ccccc1', 'N=Cc1cn([C@@H]2CCC[C@]2([NH+]2CCCC2)c2ccccc2)nc1C(F)F', 'N=Cc1cnc(C2CC2)nc1-c1cccc([N+](=O)[O-])c1', 'N=Cc1cnn(-c2ccc(F)cc2F)c1NC(=O)C(=O)NC(=O)c1ccc2ccccc2c1', 'Oc1cc(O)cc(C(=O)N2CCN(S(=O)(=O)Cc3ccc4c(c3)CCCC4)CC2)c1', 'Oc1cc(O)cc(Oc2ccc(C(=O)NC3CC3)cc2Cl)c1', 'Oc1cc(O)cc(N2CCN(C(=O)c3ccc(COc4ccccn4)cc3)CC2)c1', 'Oc1cc(O)cc(Cl)c1C[NH2+]Cc1ccc(Cl)nc1', 'Oc1cc(O)cc(/C=C(/C(=O)NCCc2ccccc2Cl)C(F)(F)F)c1', 'Oc1cc(OCOCC2CCN(c3ccccc3)CC2)ccn1', 'Oc1cc(OCOc2ccc(Br)cc2)[nH]n1', 'Oc1cc(Br)cs1\n\n', 'Oc1cc(O)c2nonc2c1N[C@H]1CC[NH2+]C1', 'Oc1cc(O)c2nc[nH]c2c1', 'Oc1ccc(/C=N\\c2ccccc2)c[nH+]1', 'Oc1ccc([N+](=O)[O-])c(CC[NH2+]C2CC2)c1Cl', 'Oc1ccc([N+](=O)[O-])c(C2=NC=CCS2)c1[N+](=O)[O-]', 'Oc1ccc([N+](=O)[O-])c(CN2C[C@@H](O)C[C@H]2c2ccccc2)c1', 'Oc1ccc([N+](=O)[O-])c(CSc2nnc[nH]2)c1Cl', 'Oc1ccc([N+](=O)[O-])c(C[NH+]2CCC(Oc3ccccc3)CC2)c1', 'Oc1ccc([N+](=O)[O-])c(C[NH2+]Cc2ccccc2Br)c1', 'O=C(/N=C/N=C/c1cccc(Cl)c1)NCCOc1ccc(F)cc1', 'O=C(/N=C/NCC1CCCC1)c1ccc(F)cc1F', 'O=C(/N=C/Nc1ccc(NSc2ncccc2F)cc1OC(F)F)Nc1ccc(C(N)=O)cc1', 'O=C(/N=C/N1CCN(Cc2ccco2)CC1)C(F)(F)F', 'O=C(c1cccs1)C1CCN(c2nc3ccccc3oc2=O)CC1', 'O=C(c1cccs1)c1cnn(-c2ccc(F)cc2)c1', 'O=C(c1cccs1)N1CCC[C@@H](C(=O)OCc2ccc(C(F)(F)F)cn2)C1', 'O=C(c1cccs1)[C@H]1Cc2ccccc2CN1C(=O)c1ccco1', 'O=C(c1c[nH]c([O-])n1)N1CC(c2nc(-c3ccco3)nn2CC(F)F)c2ccccc21', 'O=C(c1ccco1)N1CCN(c2cc[nH+]cc2)CC1', 'O=C(c1ccc[nH]c1=O)N1CCO[C@@]2(CCS(=O)=C2CO)N1', 'Oc1ccccc1[C@@H](CO)Nc1ccccc1OCc1cccs1', 'Oc1ccccc1[C@@H](c1cccnc1)N1CCN(S(=O)(=O)c2ccc(Cl)cc2F)CC1', 'Oc1ccccc1[C@@H](O)Cn1cc(Br)cn1', 'Oc1ccccc1[C@@H](NS(=O)(=O)c1ccc(F)cc1)c1cc(F)ccc1F', 'Oc1ccccc1[C@@H](Cl)C1CC1', 'Oc1ccccc1[C@@H]([O-])CCCNC(=O)[C@H]1COc2ccccc2O1', 'Oc1ccccc1[C@@H]([NH+]c1c(Br)cccc1Cl)[C@@H]1CCCO1', 'Oc1ccccc1[C@@H](Br)[C@@H](CO)Cc1ccccc1', 'Oc1ccccc1[C@@H]([NH2+]Cc1ccco1)c1ccccn1', 'Oc1cccc(F)n1\n\n\n\n\n', 'Oc1cccc([C@H](CNS(=O)(=O)c2ccc(Cl)cc2)[NH+]2CCCCC2)c1', 'Oc1cccc([C@H](c2[nH]nc3ncccc23)[NH+]2CCC(O)CC2)c1', 'Oc1cccc([C@H](OCC[NH3+])C(=O)[O-])c1', 'Oc1cccc([C@H](NC(=O)Cn2cncn2)c2cccs2)c1', 'Oc1cccc([C@H]([NH2+]C2CCCC2)C2CC2)c1', 'Clc1ccc(C2CC2)cc1F', 'Clc1ccc(C2(OCc3ccc(-n4cnnc4)cc3)CCC2)cc1', 'Clc1ccc(C2=NO[C@H](C[NH+]3CCCCC3)C2)cc1', 'Clc1ccc(C2S[C@@H](CO[C@H]3OCc4ccccc4O3)C2)cc1', 'Clc1cccc(Br)c1COC1CC[NH+](CC2=c3ccccc3=[NH+][C@@H]2c2ccccc2)CC1', 'Clc1cccc(CCn2c(-c3cc(Cl)ncc3[N-]S(=O)(=O)c3ccccc3)no2)c1', 'Clc1cccc(C(=O)OC2CC[NH+](Cc3ccccc3Cl)CC2)c1', 'Clc1cccc(C2=NO[C@H](c3cccnc3)C2)c1', 'Clc1cccc(COc2cccc(/C(Nc3ncccn3)[NH+]3CCCCCC3)c2)c1', 'Clc1cccc(CN2CCCC[C@H]2c2nc3ccccc3o2)c1', 'Clc1cccc(Cn2c3ccccc3nc2SCc2ccc(Br)cc2)c1', 'Clc1cccc(C[NH2+]Cc2cccc3c2OCCO3)c1', 'N=Cc1nonc1Cc1ccc(Cl)cc1', 'O=C(c1c[nH]c2nc[nH]c12)N1CCC2(CC1)N(c1ccccc1)CC[C@H]2c1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1[C@H]1Cc2ccccc2O1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnc2cc[nH]c12', 'Clc1nnns1\n\n\n\n', 'O=C([O-])/C(CCC(=O)Nc1ccc2ccccc2c1)c1ccc(C#N)cc1', 'O=C([O-])/C(C(=O)c1ccccc1)NCCc1ccccc1', 'O=C([O-])/C(Cc1ccccc1)=N\\NC(=O)c1ccc(C(F)(F)F)cc1', 'O=C([O-])/C(C=C(/C#N)C1CCCCCC1)NNc1ccccc1C#CCO', 'O=C([O-])/C(CO)CCn1nnc2ccccc21', 'O=C([O-])/C(CNC(=O)c1cc(-c2cccs2)on1)c1ccccc1', 'O=C([O-])/C(C#N)[C@@H](c1ccccc1Cl)c1c[nH]c2ccccc12', 'O=C([O-])/C(CSc1ccccc1)C(=O)Nc1ccccc1', 'OC[C@@H]1c2ccccc2C(=O)N1C[C@@H]1CCCO1', 'OC[C@@H]1[C@H]([NH2+]C2CC2)CCC[C@@H]1CC[NH+]1CCCCC1', 'O=[N+]([O-])c1c(O)sc2c1nnn2-c1ccccc1', 'O=[N+]([O-])c1c(O)ccc2c1NC(Cc1ccccc1Cl)C2=O', 'O=[N+]([O-])c1c(O)nc2ncnc(NC3CCC3)c2c1C[NH+]1CCCCC1', 'Clc1ccsc1F\n\n', 'Clc1ccsc1/N=C/c1cccc(-c2c(F)cccc2F)c1', 'Clc1ccsc1NC(=O)Cc1c[nH]nc1Cc1ccccc1', 'Clc1ccsc1Nc1nc(-c2ccc(Br)s2)no1', 'Clc1ccsc1N1CCCCC[C@@H]1C(=O)[O-]', 'Clc1ccsc1N[C@@H]1CCCC[C@H]1COc1ccccc1F', 'Clc1ccsc1NS(=O)(=O)c1ccc(C(=O)[O-])cc1', 'Clc1ccsc1N[C@H]1CCOc2c(F)cccc21', 'Clc1ccsc1N/N=C\\c1c[nH]c2ccccc12', 'Clc1cc(Cl)nc(-n2ccnc2)c1', 'Clc1cc(Cl)nc2[nH]nc(C(=O)C[S@](=O)CC3CC3)c12', 'Clc1cc(Cl)ncc1O[C@@H](C(=O)c1ccccc1)c1ccccn1', 'O=C(c1cn(Cc2cccnc2)c2c1CCCC2)N1CCCCCCC1', 'Clc1nnnc(C(=O)Oc2ccccc2F)c1N', 'Clc1nnnc(Cc2ccc(Br)s2)c1N', 'Clc1nnnc(C2=NC(=O)Nc3ccccc32)n1', 'Clc1nnnc(COc2ccccc2Cn2cccn2)n1', 'Clc1nnnc(CN2C(=O)/C(=C/c3cccc4ccccc34)c3ccccc32)c1-c1ccccc1', 'Clc1nnnc(CI)c1', 'Clc1nnnc(CSc2nncn2-c2ccccc2)n1', 'Clc1nnnc(C[NH3+])c1Cl', 'Clc1cc(Br)cc(CCOCc2ccc(F)cc2)c1', 'Clc1cc(Br)ccc1S[C@@H]1CCOC([O-])C1', 'Clc1cc(Br)cc2nnnn12', 'Clc1nnn(C[C@@H](CO)Oc2ccccc2)c1CN1CCOCC1', 'Clc1nnn(C[C@@H]2CCC[NH+](C3CCCC3)C2)c1Br', 'O=[N+]([O-])c1c(Cl)ncc(Cl)c1Cl', 'O=[N+]([O-])c1c(Cl)nccc1C1CC1', 'O=[N+]([O-])c1c(Cl)ncc2cc(S(=O)(=O)Nc3ccc(Br)cc3F)ccc12', 'COc1ncncc1-c1ccc(Br)c(S(=O)(=O)N2CCOCC2)c1', 'COc1ncncc1C(=O)Nc1ccnn1C1CCCCC1', 'COc1nc2c3c(sc2n1CCCN(C)C(=O)C[S@@](=O)C)CC3', 'COc1ncncc1CCCNC(=O)c1cc(-c2ccco2)on1', 'COc1ncncc1C(=O)Nc1ccc([N+](=O)[O-])cc1', 'COc1ncncc1C[S@@](=O)Cc1nc(-c2cccs2)no1', 'COc1ncncc1CNC(=O)C(=O)Nc1cccc(OC)c1', 'COc1ncncc1C[C@@H]([NH3+])Cc1cc(Cl)ccc1Cl', 'COc1ncncc1CSCOC(=O)Cc1ccccc1', 'COc1ncncc1C[NH+]1C[C@H](C(N)=O)[C@@H]2CCCC[C@@H]21', 'C=C1[C@H](C)C[C@H](C(=O)O[C@@H](C)c2ccccc2)[C@H]1O', 'C=C1[C@H]2CC[C@@H]1CN(Cc1nc3ccccc3s1)CC2', 'Oc1cc(Br)c2s/c(=C\\c3ncc(C(F)(F)F)o3)c(=O)c3ccccc3c2c1O', 'Oc1cc(Br)c2sccc2c1Oc1c(F)cc(F)c(F)c1F', 'Oc1cc(Br)c2sncc2c1F', 'Clc1ccsc1F\n\n\n', 'Clc1ccsc1Sc1cccc(F)c1', 'Clc1ccsc1/N=C/c1cccc(OC2CCCC2)c1', 'Clc1ccsc1/N=N/CCOc1ccccc1Cl', 'Clc1ccsc1/N=c1\\sccn1Cc1cccc([N+](=O)[O-])c1', 'Clc1ccsc1[C@@H](CO)CC[C@H]1CCCCO1', 'Clc1ccsc1[C@@H](O)c1ccncc1', 'Clc1ccsc1[C@@H](N[NH+]1CCC[C@@H](O)C1)c1ccccc1', 'Clc1ccsc1[C@@H](F)C(=O)[O-]', 'Clc1ccsc1[C@@H](Cl)Cc1cc(Cl)ccc1F', 'Clc1ccsc1[C@@H](Br)c1ccc(Br)cc1', 'Clc1ccsc1[C@@H]([NH3+])c1ccccc1', 'Clc1ccsc1[C@@H]([NH2+][C@@H]1CCCc2cccnc21)C1CC1', 'N#C[C@@H]1CN(Cc2ccccc2Cn2cccn2)CCS1', 'N#C[C@@H]1OC(=O)N(CCCN2CCc3ccccc32)C1=O', 'N#C[C@@H]1[C@H](c2cccc(Br)c2)[C@@H](c2ccccc2)OC1=O', 'O=C([C@H](Cl)c1ccccc1F)N1CCN(c2ccccc2)CC1', 'O=C([C@H](Cl)c1[nH]ccc1C(F)(F)F)[C@@H]1CO[C@@H]1c1ccccc1', 'O=C([C@H](Cl)c1ccccc1)N1CCOCC1', 'O=C([C@H](Cl)c1nc2ccccc2s1)[NH+]1CCCCC1', 'O=C(Cc1noc(Cc2ccc(Cl)cc2)n1)NCCc1ccccc1Br', 'O=C(Cc1nonc1CO)NCCc1ccccc1', 'O=C(CN(c1ccccc1)S(=O)(=O)c1ccc(F)cc1)NC[C@@]1(O)CCc2ccccc21', 'O=C(Cc1nc(-c2ccccc2)no1)NCc1cc(Cl)cc2ccccc12', 'O=C(Cc1ncccc1O[C@H]1CCSc2ccc(Cl)cc21)c1ccccc1', 'O=C(Cc1nc2ccccc2c(=O)[nH]1)Nc1ccc2ccccc2c1', 'O=C(Cc1ncon1)Nc1ncc(Br)cc1Cl', 'O=C(Cc1nc[nH]n1)Nc1c(Br)cc(Br)s1', 'O=C(/N=C\\c1c(F)cccc1Cl)c1cccc2cccnc12', 'O=C(/N=C\\c1c[nH]c2ccccc12)c1ccc([N+](=O)[O-])cc1', 'O=C(/N=C\\c1ccc(S(=O)(=O)N2CCCC2)cc1)c1ccccc1Cl', 'O=C(/N=C\\Nc1cccc(S(=O)(=O)N2CCOCC2)c1)C1CCC(c2ccccc2)CC1', 'O=[N+]([O-])c1c(Cl)n2nnc(-c3ccncc3)c2cc1Cl', 'O=[N+]([O-])c1c(-c2nc(C3CC[NH2+]CC3)co2)cc(O)c2c1CCN(C(=O)[C@@H]1COc3ccccc3O1)C2', 'O=[N+]([O-])c1c(-c2nc3ccccc3o2)ncnc1N1CCOCC1', 'O=[N+]([O-])c1c(-c2ncnn2Cc2ccccc2)cc2ccccc2n1', 'O=C(CNc1nn(Cc2cc(F)cc(Cl)c2)c(=O)c2occc12)Nc1cccnc1Cl', 'O=C(CNc1nnc(Cl)c2ccccc12)Nc1ccccc1Cl', 'O=C(CNc1nn2nc(-c3cccc(F)c3)oc2c1Cl)Nc1cc([N+](=O)[O-])ccc1F', 'O=C(c1c[nH]c2ncn(-c3ccccc3)c12)c1ccccc1', 'O=C(c1c[nH]c2ncn(Cc3ccccc3)c12)N1CCCCCC1', 'O=C(c1c[nH]c2ncn([C@H](CO)c3nc4ccccc4cc3)c12)N1CCN(C(=O)c2ccccc2)CC1', 'O=C(CNCc1csc(-c2ccccc2)n1)c1ccccc1O', 'O=C(CNCc1c(F)cccc1F)N1C[C@H](O)[C@H](O)C1', 'O=C(CN(c1cnccc1)S(=O)(=O)[O-])C1CCCC1', 'O=C(CS(=O)(=O)c1ccccc1)NCC1(C2CC2)CCCCC1', 'O=C(CN(c1cn[nH]c1)c1ccccc1)N1CCN(Cc2cccs2)CC1', 'O=C(CN(c1cncnc1)S(=O)(=O)c1ccc(Br)cc1)N1CCCCN1', 'O=C(CN(c1cnn(C(C)C)n1)c1ccccc1)N1CCN(C2CC2)CC1', 'O=C(CS(=O)(=O)CCN1C(=O)/C(=C/c2ccc(C#N)cc2)SC1=S)Nc1ccccc1', 'O=C(CS(=O)(=O)C[C@H]1CCCCO1)NN1CCOCC1', 'O=C(CS(=O)(=O)Cc1cccc(Cl)c1)Nc1c(F)cccc1F', 'O=C(COc1ccc([N+](=O)[O-])cc1)N[C@@H]1CCCN(c2ccc(F)cc2)C1', 'O=C(COc1ccc(CN2CCOCC2)cc1)N[C@@H]1C[C@H]2CCCC[C@@H]2C1', 'O=C(COc1ccc(NC(=O)C[C@H]2CSc3ccccc32)[nH]1)Nc1ccccc1', 'O=C(COc1ccc(I)cc1)Nc1ccnc(C(=O)[O-])c1', 'O=C(COc1ccc(F)c(F)c1)N1CCC(Oc2ccc(C(C)=O)cc2)CC1', 'O=C(COc1ccc(S(=O)(=O)N2CCCCCC2)cc1)NNC(=O)CCl', 'O=C(COc1ccc(Cl)cc1)NCC1([NH+]2CCCC2)CC(C)CC1', 'O=C(COc1ccc(Br)cc1)NCC[NH+]1CCCCC1', 'O=[N+]([O-])c1c(Cl)n2nnnc2n1Cc1ccc(F)cc1', 'OC[C@@]1([NH3+])CC[C@@H](CNc2ccccc2Br)C1', 'Oc1ccc(/C=C/NC(=O)CSc2nc3ccccc3nc2N)c(Cl)c1', 'Oc1ccc(/C=N\\NC2=NC=NN2)cc1', 'Oc1ccc(/C=N\\N(Cc2ccccc2)S(=O)(=O)c2ccccc2)cc1F', 'Oc1ccc(/C=N\\Nc2nc(Cl)ccn2)cc1', 'Oc1ccc(/C=N\\N=C2\\NCCS2)cc1N1CCCCCC1', 'Oc1ccc(/C=N\\N[C@@H]2CCC[C@@H]2C[C@@H]2CCOC2)cc1', 'O=C(c1cc(Cl)ccc1[N+](=O)[O-])N1CCN(S(=O)(=O)c2cccc([N+](=O)[O-])c2)CC1', 'O=C(c1cc(Br)cc(Cl)c1)[C@@H]1CCOC1', 'O=C(c1cc(Br)c2c(c1)OCO2)N1CCC(Oc2ccccc2)CC1', 'O=C(c1cc(Br)cnc1Br)N1CCN(C(=O)c2cccs2)CC1', 'O=C(c1cc(Br)cs1)N1CCN(c2ccccc2C(F)(F)F)CC1', 'O=C(c1cc(Br)c[nH]1)N1CCC[C@H](CO)C1', 'O=C(c1ccsc1C1CC1)N1CCCCC[C@@H]1c1ccc(F)cc1', 'O=C([C@H](NSCc1cc(F)c(F)c(F)c1)N1CCOCC1)NCC1(CO)CC1', 'O=C([C@H](NS(=O)(=O)c1ccc([N+](=O)[O-])cc1)c1cccs1)Nc1ccccc1C(F)(F)F', 'O=C([C@H](NSc1ccccc1)[C@H]1CSC(=O)O1)c1ccccc1', 'Oc1cc(OC2=NN(c3ncccn3)C[C@@H]2c2ccccc2)ccc1OCCOc1ccccc1F', 'Oc1cc(OC2CC[NH2+]CC2)ccc1Oc1cccc(Br)c1Br', 'Oc1cc(C=NC(=O)c2cccc(Br)c2)ccc1OC(F)F', 'Oc1cc(C=NOc2[nH]nc(Nc3ccccc3)n2)ccc1O', 'Oc1cc(C=NNCc2ccc(CN3CCOCC3)o2)c(F)cc1Cl', 'Oc1cc(C=N[C@@H]2CC[NH+]=C2c2ccco2)ccc1O', 'Oc1cc(C=NSCc2ccsc2)nc2sccc12', 'Oc1cc(OCOCCOc2ccc(C(F)(F)F)cc2)ccc1Br', 'Oc1cc(OCOCc2nc3ccccc3s2)ccc1N1CCOCC1', 'Oc1cc(OCOC2CCCC2)c(Br)c(Cl)c1OO', 'Oc1cc(OCOC[C@@]2(CBr)C3=CCC[C@@H]2C3)ccc1F', 'O=C(/C=C/O)N1CCN(S(=O)(=O)c2c(Cl)cccc2Cl)CC1', 'O=C(/C=C/O)C[C@H](c1ccccc1)NC(C)(C)C', 'O=C(/C=C/O)O[C@@H]1CCc2cc(OC(=O)[C@@H]3CCCCC[C@@H]3S(C)(=O)=O)ccc21', 'O=C(/C=C/O)c1ccc(Cl)cc1', 'O[C@@H](Sc1ccccc1)C(=O)N1CCCCC1', 'O[C@@H](Sc1cnn(CCO)c1)C(F)(F)Cl', 'Oc1ccc(F)c(F)c1CNC[C@@H](O)c1ccc(Cl)cc1', 'Oc1ccc(F)cc1-c1cccc(F)c1O', 'Oc1ccc(F)cc1-n1nc(C(C)(C)COC2CCCCC2)c2ccccc21', 'Oc1ccc(F)c([C@@H](C[NH2+]C(N)=O)[NH+]2CCCC2)c1', 'Oc1ccc(F)c([C@@H]2CSCCN2C(=O)C2=CCCCC2)c1', 'NS(=O)(=O)N1CCN([C@H]2CCOC3(CCCC3)C2)CC1', 'NS(=O)(=O)N1C(=O)/C(=N/c2nc(-c3ccccc3Br)n[nH]2)c2ccccc21', 'NS(=O)(=O)N1Cc2ccc(OC(=O)c3cc(C#N)ccc3Cl)cc2C1', 'O=C(c1c[nH]nc1C[NH+](CC1CC1)C1CC1)N[C@H]1CCN(c2ccc([N+](=O)[O-])cc2)C1=O', 'O=C(c1c[nH]nc1C[NH+]1CCC[C@@H]1c1nccs1)N1CCCC1', 'N[C@@H]1c2ccccc2O[C@@H]1C(=O)N/N=C/[C@@H]1CCCC2=Cc3ccccc3N21', 'N[C@@H]1c2ccccc2[C@@H]2OCC[C@H]21', 'N#Cc1c([N+](=O)[O-])c2cc([N+](=O)[O-])ccc2n1Cc1ccc(F)cc1', 'N#Cc1c(O)cc(N)cc1C(=O)Nc1cccc(OCO)c1', 'N#Cc1c(NC(=O)C=CCCS)c2ccccc2n1C', 'N#Cc1c(F)cc(CNC(=O)NC[C@H]2CC=CCC2)cc1C(F)(F)F', 'N#Cc1c(-c2ccccc2)cc(C(=O)N[C@@H]2CC(=O)N(Cc3ccccc3)C2)n1C1CC1', 'N#Cc1c(SCCC(=O)Nc2ccc(Br)cc2)n[nH]c1NC(C)C', 'N#Cc1c(Cl)cc(Cl)cc1C(=O)NCc1cccc(Br)c1', 'N#Cc1c(Br)cccc1N[C@H]1CCSC1=O', 'N#Cc1c(/C=C\\C(=O)c2cccc(F)c2)cn[nH]/c1=N\\OCCc1ccccc1', 'N#C/C(=C\\c1cn(CC(=O)[O-])c(C(=O)N2CCOCC2)c1C#N)c1ccco1', 'N#C[C@@]1(C2CC[NH+](CC(=O)N[C@H]3CCOc4ccc(F)cc43)c3[nH]cnc3C2)CC1', 'N#C[C@@]1(c2cc[nH]c2)CN(CC(F)(F)F)c2ccccc21', 'N#C[C@@]1(NC(=O)CN2CCCC2)CC1', 'N#C[C@@]1(SCC(F)(F)F)CSCCO1', 'N#C[C@@]1([C@H](O)c2ccco2)CCC1(C)C', 'N#C[C@@]1([NH+]2CCC(n3cncn3)CC2)CCCSC1', 'N#C[C@@]1([NH2+]C2CC2)CCC[C@@H]1CCSc1ccccc1', 'ClCc1csc2nc3c(cc12)CNC3', 'CC[S@](=O)Cc1cc(NC(=O)[C@@H]2CCCN2C(=O)N2CCCC2)cc(F)c1', 'CC[S@](=O)[C@H]1CCC[C@@H](NC(=O)N(CCC(F)(F)F)C2CCCC2)C1', 'CC[S@](=O)[C@@H]1CCCC[C@@H]1NC(=O)N1CCC(OCc2ccc(Cl)cc2)CC1', 'ClCc1cccc(C2n3c(cc4ccccc4c3=O)C2)c1', 'ClCc1ccccc1-n1cccc1', 'ClCc1cccc2c1O[C@@H](Cc1cc(Br)ccc1OC[C@@H]1CCCO1)C2', 'ClCc1csc(NCc2cccc(OCC3CCOCC3)c2)n1', 'ClCc1csc(Nc2ccc(OC(F)F)cc2)c1', 'ClCc1csc(N2CCO[C@@H](c3ccccc3)C2)c1Cl', 'ClCc1csc(N[C@H](CO)CO)n1', 'ClCc1ccnnc1N[C@H](c1ccccc1)c1ccccc1', 'ClCc1cc(Br)cnc1OCCn1cccnc1=O', 'ClCc1csc2nc[nH]c(=O)c12', 'ClCc1csc2nc(SCc3ccc(Cl)cc3)nn12', 'ClCc1csc2ncn(Cc3ccc(Oc4ccccc4)cc3)c12', 'ClCc1ccnn1-c1ccc([N+](=O)[O-])cc1', 'ClCc1ccnn1Cc1ccc(F)cc1O', 'ClCc1cc(F)cc(N2CCNCC2)c1', 'ClCc1ccc[nH]c1=O', 'ClCc1ccc[nH]1', 'ClCc1csc(OCc2ccccc2)n1', 'ClCc1csc(Oc2cccc(F)c2)n1', 'ClCc1csc(O[C@@H]2CCOC2)n1', 'ClCc1cc(OC[C@H]2CCCO2)nc(OC2CCCCC2)n1', 'ClCc1cc(O)cc(F)c1I', 'ClCc1cc(Oc2c(Cl)cccc2Cl)ncn1', 'ClCc1cc(O[C@@H]2CCc3ccccc32)c(F)cc1F', 'ClCc1ncoc1Cn1c(=O)oc2ccccc21', 'ClCc1nnsc1-c1ccc(Cl)c(Cl)c1O', 'ClCc1ncoc1[C@@H]1CCC[NH+](Cc2ccc(F)cc2)C1', 'ClC[C@H]1CNCC[C@H]1C[NH+]1CCCC1', 'ClC[C@H]1CN(Cc2cccc(O)c2)CCO1', 'ClC[C@H]1CNc2ccc(Cl)cc21', 'ClC[C@H]1CN2CCC[C@@H]2CO1', 'ClC[C@@H]1C[C@@H](O)c2ccccc21', 'ClC[C@@H]1C[C@@H]1c1cc(F)cc(F)c1', 'ClCc1ccnnc1', 'ClCc1ccnnc1Cn1ccc2ccccc21', 'ClCc1ccnnc1NC[C@H]1COc2ccccc2O1', 'ClCc1ccnnc1-n1cncn1', 'ClCc1ccnnc1SCC(=O)NC[C@@H]1CCCO1', 'ClCc1ccnnc1Cl', 'ClCc1csc(C(=O)N2CCN(Cc3ccccc3Cl)CC2)c1', 'ClCc1csc(Cc2cccs2)n1', 'ClCc1csc(C2CC2)n1', 'ClCc1csc(COc2ccc(Br)cc2)n1', 'ClCc1csc(CN2CCN(c3ccc(Oc4ccccc4)nn3)CC2)c1', 'ClCc1csc(C[C@@H]2CCCOC2)n1', 'ClCc1csc(Cn2ccnc2-c2ccccc2F)c1', 'ClCc1csc(C[NH2+]C2CCCC2)n1', 'ClCc1nnc(CNS(=O)(=O)[C@H]2CCCc3ccccc32)s1', 'ClCc1nnc([C@@H]2COc3ccc(Cl)cc3C2)n1CC1CC1', 'ClCc1nnc(-c2cccs2)o1', 'ClCc1nnc(Sc2ccc(Cl)cc2)[nH]1', 'ClCc1nnc([C@]2(CC(F)(F)F)N=CN2C(=O)c2ccsc2)s1', 'ClC[C@@H]1C(=O)CCC(C2CC2)C1', 'ClC[C@@H]1C(C#N)C=CC[C@@]1(O)c1ccccc1', 'ClCc1ccc2c(c1)CCC[C@@H]2[NH2+][C@@H]1CCSc2ccccc21', 'ClCc1ccc2nc[nH]c(=O)c2c1', 'ClCc1cc(CCO)ccc1F', 'ClCc1cc(CO)oc1[O-]', 'ClCc1cc(CNc2cccc(C(F)(F)F)c2)n[nH]1', 'ClCc1cc(C#N)ccc1Oc1ccccc1', 'ClCc1cnc(Cl)n1Cc1ccccc1', 'ClCc1cncn2cnnc12', 'ClCc1cncc(-c2ccc(O)c3ccccc23)n1', 'ClCc1cnc2cc(Br)ccc2c1NC1CCCCC1', 'ClCc1csc([C@H]2O[C@@H]3CCN(c4ccc(F)cc4)CCC[NH2+]CCC[C@@H](O)[C@H]32)n1', 'ClC[C@H]1OCc2ccccc21', 'ClC[C@H]1OCC[C@@H]1N1CCc2ccccc2C1', 'ClC[C@H]1OC(c2ccccc2)=C1Cl', 'ClC[C@H]1OC[C@H]2CCCC[C@@H]2C1', 'ClCc1nc(C[NH+]2CCN([C@H](CO)c3ccccc3)CC2)cs1', 'ClCc1nc(Oc2ccc(Cl)cc2)nnc1-c1cccc(Cl)c1', 'ClCc1nc(N2CCCCC2)c2nc(N3CCCCC3)nnc2n1', 'ClCc1nc(I)cn1C1CC1', 'ClCc1nc([C@@H]2CCCN(C(=O)[C@@H]3CCCO3)C2)cs1', 'ClCc1nc(-c2ccc([O-])cn2)cs1', 'ClCc1cnn(CCC(=O)c2ccc(Cl)cc2)c1-n1cccn1', 'ClCc1cnn([C@H]2CCCCC[C@@H]2C(=O)N2CCCC2)c1', 'ClCc1cnn([C@@H]2CCCC[NH+]2Cc2ccccn2)n1', 'Clc1nnc(-c2ccccc2)n1COc1ccccc1', 'Clc1nnc([S-])n1-c1cccc(F)c1', 'Clc1nnc(Cl)[nH]c1=S', 'ClC[C@@H]1COCCN1c1cccc(O)c1', 'ClC[C@@H]1CO[C@H](c2ccccc2F)N1', 'ClC[C@@H]1COc2ccc(F)cc21', 'ClC[C@@H]1CO[C@@H](c2ccccc2)[NH+](CC2CC2)C1', 'Cc1nonc1CC(=O)N1CCCC[C@@H]1C(=O)[O-]', 'Cc1nonc1NC(=O)/C=C/c1cc(C)cc([N+](=O)[O-])c1', 'COC[C@@H](O)CN1CCN(C(=O)NCCc2ccc(F)cc2)CC1', 'COC[C@@H](C)NC(=O)c1ccnc(OCC(C)C)c1', 'COC[C@@H](NC(=O)OC(C)(C)C)C(=O)NN', 'COC[C@@H]([NH2+]CC1CCN(Cc2ccccc2F)CC1)C(=O)OC', 'COCc1c(-c2ccccc2)[nH]c(=S)n1C', 'COCc1c(C)sc2ncnc(N/C(=N/O)S(C)(=O)=O)c12', 'COCc1c(N)[nH+]c(C)n1CC[NH+]1CCC[C@H]1CC', 'COCc1c([O-])n2cc(-c3ccccc3)cc(-c3ccc(Cl)cc3)c2n1', 'Cc1nonc1NC(=O)N1CCN(c2cc(C)ccn2)CC1', 'Cc1nonc1N(C)C(=O)[C@@H](C)Oc1ccccc1Cl', 'Cc1nonc1N[C@H](C)C(=O)NCc1ccccc1Br', 'Cc1nnsc1C(=O)N1CCOCC[C@@H]1C', 'CCO[P@](C)(=O)c1ccc(Cl)c([N+](=O)[O-])c1', 'CCO[P@]1(=O)[C@@](C)(C(=O)OC)[C@@]1(O)C(F)(F)F', 'Oc1cc(F)ccc1C[NH2+]Cc1ccc(F)cc1F', 'OC[C@H](CCS(=O)(=O)c1ccccc1)N[C@H]1CC(=O)N(c2ccccc2)C1', 'OC[C@H](C(=O)N1CCN(c2nc(-c3ccccc3)cs2)CC1)c1cccs1', 'OC[C@H](Cc1ccccc1)Oc1ccc(N2CCOCC2)[nH+]c1', 'OC[C@H](C1CC[NH2+]CC1)N1CCOCC1', 'OC[C@H](CO)Nc1ncnc2c1c(=O)n(C)c1ccccc12', 'OC[C@H](CNS(=O)(=O)c1cccs1)c1ccccc1', 'OC[C@H](C[C@@H]1CCC[NH2+]1)c1nc(-c2ccccc2)no1', 'OC[C@H](Cn1ccc(=O)[nH]c1=O)Nc1ccc2c(c1)OCO2', 'Oc1ccc(Oc2cc(C(=O)[O-])cc([N+](=O)[O-])c2)cc1', 'Oc1ccc(Oc2ccc(C(=O)Nc3ccccc3)c(-c3ccccc3)n2)cc1', 'Oc1ccc(Oc2ccc(NCC(=O)N3CCCc4ccccc43)cc2)cc1', 'Oc1ccc(Oc2ccc3ccccc3n2)cc1Cl', 'Oc1ccc(Oc2ccccc2C[NH2+]C2CC[NH2+]CC2)cc1', 'Oc1ccc(O[C@@H](c2ccco2)[C@H](CO)C2CC2)cn1', 'Oc1ccc(F)cc1CC1CCN(C(Cc2ccccc2)Cc2ccco2)CC1', 'Oc1ccc(F)cc1C(=O)Nc1ccc([N+](=O)[O-])cc1', 'Oc1ccc(F)cc1Cc1c[nH]c(Nc2ccccc2Cl)n1', 'Oc1ccc(F)cc1COc1ccccc1', 'Oc1ccc(F)cc1CN1CC[NH+]c2ccccc2c1=O', 'Oc1ccc(F)cc1C[C@@H]([NH3+])c1ccccc1F', 'Oc1ccc(F)cc1Cn1cnc(C[NH2+]C2CCOCC2)n1', 'Oc1ccc(F)cc1CSC1=N[C@@H](c2ccc(F)c(F)c2)N1C1CCC1', 'Oc1ccc(F)cc1C[NH+]1CCN(c2cc[nH+]cc2Cl)CC1', 'Oc1ccc(F)cc1C[NH2+]Cc1cc(Cl)ccc1O', 'O=[N+]([O-])c1c(Cl)nc(C2CCCCC2)ncn1C1CC1', 'O=[N+]([O-])c1c(Cl)nc(N2CCCC2)nc1Oc1ccc(Cl)cc1', 'O=[N+]([O-])c1c(Cl)nc([C@@H]2CCCC[C@@H]2O)nc1-c1ccccc1', 'O=[N+]([O-])c1c(Cl)nc(-c2ccc(OS(=O)(=O)C3CCCC3)cc2)c2ccccc12', 'O=[N+]([O-])c1c(Cl)nc(Cl)cc1Br', 'O=[N+]([O-])c1c(Cl)nc(Br)s1', 'Oc1cc(-n2cc[nH+]c2)ccc1/N=C/c1ccccc1[N+](=O)[O-]', 'Oc1cc(F)cc2[nH]ncc2c1=O', 'Oc1cc(F)cc2c1cnn2Cc1cccs1', 'Oc1cc(F)cc2ncccc12', 'Clc1nncnc1N', 'Clc1nncnc1NC1(c2ccccc2)CCOCC1', 'Clc1nncnc1Nc1ccc([N+](=O)[O-])cn1', 'Clc1nncnc1N1CCCC[C@@H]1Cn1cncn1', 'Clc1nncnc1N=C\\c1ccc(Cl)cc1Cl', 'Clc1nncnc1N[C@@H](c1ccccc1)c1ccccn1', 'Clc1nncnc1N[C@H](CO)c1ccccc1F', 'Clc1nnc(Br)c(Br)c1C(=O)[O-]', 'Clc1nnn(-c2nccs2)n1', 'Clc1nnn(-c2ncc(C(F)(F)F)cc2Cl)c1Br', 'Clc1nnn(-c2ncc3nnn(-c4ccccc4)c3c2)c1N', 'Clc1nnn(-c2ncccc2[N+](=O)[O-])c1I', 'Clc1nnns1\n\n\n\n\n', 'O=C(Cc1c(O)cccc1Cl)N1CCC[C@H](O)C1', 'O=C(Cc1c(N)nc(-c2ccccc2)o1)c1ccccc1', 'O=C(Cc1c(F)cccc1F)Oc1ccc2c(c1)OCCO2', 'O=C(Cc1c(-c2ccc(Cl)cc2)cccc1O)Nc1ccc(C(F)(F)F)cc1', 'O=C(Cc1c(Cl)cccc1Cl)Nc1c(Cl)ccc2cccnc12', 'O=C(Cc1c([O-])c2ccccc2oc1=O)Nc1ccc(Oc2ccccc2)cc1', 'O=C([O-])[C@@H]1CC(c2ccccc2)=Nc2ncnn21', 'O=C([O-])[C@@H]1Cc2cc(-c3cccs3)[nH]c2N1', 'O=C([O-])[C@@H]1CO[C@H](c2ccccc2)S1', 'O=C([O-])[C@@H]1CN([C@@H]2CCCc3c(F)cccc32)C1', 'O=C([O-])[C@@H]1C[C@@H]2CC[P@]1[C@H]1O[C@@H]1[C@@H](c1ccc(Cl)cc1)[C@@H]1C[C@H]1O2', 'O=C([O-])[C@@H]1CSCN1C(=O)Cc1ccccc1Cl', 'O=C([O-])[C@@H]1C[C@H]1c1cccc(Cl)c1Cl', 'O=C([O-])[C@@H]1C[C@](O)([C@H](F)C(F)(F)F)N(Cc2ccc(Cl)cc2)C1', 'COc1nnnc(Sc2nnc(C)c(C)n2)c1C', 'COc1nnnc(NC(=O)CN2CC(=O)Nc3ccccc32)n1', 'COc1nn(CCC(=O)N[C@@H](C)C[C@H]2CCCCO2)c2ccccc12', 'COc1nn(CC)c2ncc(C(=O)c3cc4ccccc4o3)nc12', 'COc1nn(CC2CCCC2)c(C)c1S(=O)(=O)N(C)CCC(=O)C(F)(F)F', 'COc1nn(CC[NH+]2CCC[C@H](C(=O)NC)C2)c(C)c1Br', 'Cc1cn(CC(=O)N(CCN2CCOCC2)Cc2ccccc2F)nc1[O-]', 'Cc1cn([C@@H]2CCCCCN2c2ccccc2)c(C#N)c1N', 'Cc1cn([C@H](C)CNC(=O)C[C@H](O)c2ccoc2)n1C', 'Clc1ccc(-c2csc3ccccc23)cc1', 'Clc1ccc(-c2ccc(Cl)c(F)c2)cc1Br', 'Clc1ccc(-c2csc(NC[C@@H]3CCCCO3)n2)cc1', 'Clc1ccc([C@@H](Br)c2cc(F)ccc2F)cc1', 'Clc1ccc(-c2cc[nH]n2)cc1', 'Clc1ccc(-c2cc(Cn3cnnn3)ccn2)cc1', 'Clc1ccc(-c2cccc(C[NH2+]COc3ccc(F)cc3)c2)cc1', 'Clc1ccc([C@@H](OC[C@@H]2CCCC[NH2+]2)c2c[nH]c3ccccc23)cc1', 'Clc1ccc([C@@H](O)c2ccccc2)nc1', 'Clc1ccc([C@@H](CC(=O)NCCN2CCCC2)C(F)(F)F)cc1', 'Clc1ccc([C@@H](C)[NH2+][C@H]2CCCNC2=O)cc1', 'Clc1ccc([C@@H](CO)[NH2+]C2CC2)s1', 'Clc1ccc([C@@H](CNC(=O)[C@@H]2CC(=O)N(SCc3ccco3)C2)c2cnn(-c3ccccc3)c2)o1', 'Clc1ccc([C@@H](CSCC(F)(F)F)n2c(CO)nc3ccccc32)cc1', 'Clc1ccc([C@@H](C[C@H]2CCCO2)Sc2ccccc2)cc1', 'Clc1ccc([C@@H](C[NH+]2COc3ccccc3OC2)c2ccccc2)cc1', 'Clc1nncn1-c1ccccc1', 'Clc1nncn1c1c(Cl)ccc(Cl)c1Cl', 'Clc1nncn1c1cc(Cl)ccc1N1CC[C@H](CN2CCOCC2)C1', 'Clc1nncn1-n1cccc1', 'Clc1nnc(-c2ccccc2)o1', 'Clc1nnc(/C=C/c2ccccc2)n1/N=C/c1ccco1', 'Clc1nnc(/N=C/c2ccc3c(c2)OCCCO3)c(-c2ccccc2)n1', 'Clc1cc(Cl)nnc1Cl', 'Clc1cc(Cl)nn1/N=C\\c1ccccc1Cl', 'Clc1cc(Nc2ncc(I)cc2F)ccc1O', 'Clc1cc(N2CCC[C@@H]2c2nccs2)ccc1F', 'Clc1cc(Cl)cc(Cl)c1C[NH+]1CCC(O)[C@H](O)C1', 'Clc1cc(Cl)ccc1N1CCc2cc(Cl)ccc21', 'Clc1cc(Cl)cc2[nH]ncc12', 'Clc1cc(Nc2ccccc2)sc1-c1ccc(Cl)s1', 'Clc1cc(Nc2nc3ccccc3[nH]c2=O)cnc1Sc1ccc(F)cc1Cl', 'Clc1cc(Br)c2ncccc2c1OC[C@H]1CCCO1', 'Clc1cc(NSc2nnc(COc3ccccc3Cl)[nH]2)cc(Cl)c1Cl', 'Clc1cc(CSC2=NC[C@H]3CS(=O)(O)C[C@H](O)[C@H]23)ccc1O', 'Clc1cc(CSc2nncn2-c2ccc(Br)cc2)c2ccc(F)cc2c1O', 'Clc1nncn1c1c(C(F)(F)F)c(Cl)cn1C1CCC(O)CC1', 'Clc1nncn1c1c(F)cc(F)cc1F', 'Clc1nncn1c1c(-c2ccc([N+](=O)[O-])cc2)cc(Br)cc1O', 'Clc1nncn1c1c(Cl)nc(Cl)cc1Cl', 'Clc1nncn1c1c(Br)cc(I)cc1O', 'Clc1nnnc(OCCC[NH2+]Cc2ccccc2)c1[N+](=O)[O-]', 'Clc1nnnc(O)c1Br', 'Clc1nnnc(Oc2ccccc2)c1N1CCN(c2ccccc2)CC1', 'Clc1nnnc(O[C@@H]2CCCN(c3ccccc3)C2=O)c1Br', 'Clc1cc(Br)cnc1N1CCOCC1', 'Clc1cc(Br)c2nc(-c3cc(F)ccc3Cl)oc2c1', 'Clc1cc(/C=C/Cc2ccccc2F)ccc1Cl', 'Clc1cc(/C=N/N2CCOCC2)ccc1OC(=O)Cc1ccccc1', 'Clc1cc(/C=C/C(=O)N2CCN(Cc3ccccc3)CC2)ccc1O', 'Clc1cc(/C=C(\\C#N)c2cccc(Cl)c2)cc(Cl)c1Cl', 'Clc1cc(/C=Cc2ccccn2)ccn1', 'Clc1cc(/C=C2\\S/C(=C/c3ccc(O)c(OCc4ccccc4)c3)C(C#N)=C2[O-])ccc1O', 'Clc1ccccc1-c1nc(-c2ccncc2)no1', 'Clc1ccccc1[C@H](NCc1ccc(Br)cc1)c1ccccc1', 'Clc1ccccc1[C@H]1C[C@@H](c2cc(F)cc(Br)c2)[NH+](CC2CC2)C1', 'Clc1cc(Br)cn1C[C@@H]1CCCCCC1', 'Clc1cc(Br)cn1NC1CCCCC1', 'Clc1cc(Br)cn1[C@@H]1CCC[NH+](C2CCCCC2)C1', 'Clc1nncc(OCc2cccc([N+](=O)[O-])c2)n1', 'Clc1nncc(O[C@H]2CCN(c3ccccc3Cl)C2)n1', 'Clc1nncc(O)c1', 'Clc1nncc(Oc2nc(-c3ccccc3)no2)c1I', 'Clc1nncc(Cl)c1N[C@H]1COCCN1CC1CC1', 'ClC[C@H](c1ccco1)n1cncn1', 'ClC[C@H](c1nc2ccccc2s1)N1CCO[C@@H]2CCCC[C@H]21', 'Clc1nnn(C(C)(C)Cc2ccc[nH]2)c1-c1ccccc1', 'Clc1nnn(C(F)F)c1NCC(=O)N1CCN(c2ccccc2F)CC1', 'Clc1nnn(Cc2ccccc2)c1I', 'Clc1nnn(Cc2ncnn2C2CCCCCC2)c1Cl', 'Clc1nnc([S-])c2c1N=C1C[C@H](Cl)CC1(CC[NH+]1CCOCC1)C2', 'Clc1nnc([S-])=C(/C(=O)OCCN2CCOCC2)c1Cl', 'Clc1nnc([S-])n1CCc1ccccc1C(F)(F)F', 'Clc1nnc([S-])s1', 'OC[C@@H](SCc1cccc(C(F)(F)F)c1)N1CCN(C2CCN(C(=O)c3ccccc3)CC2)C1', 'OC[C@@H](S)c1cc(Br)ccc1Cl', 'OC[C@@H](Sc1nnc(-c2cccc(F)c2)n1C1CC1)c1ccc2cc[nH]c2c1', 'OCSc1nc(-c2ccco2)nc2cc(Cl)ccc12', 'ClCc1ccc[nH]c1=O', 'ClCc1ccccc1OC(F)F', 'ClCc1ccc[nH]1', 'ClCc1ccn(-c2cccc(Br)c2)n1', 'ClCc1cccc([N+](=O)[O-])c1', 'ClCc1cccc(C(=O)Oc2ccc3c(c2)CCCN3)c1', 'ClCc1cccc(OCCc2ccccc2)c1', 'ClCc1cccc(N[C@@H]2CCN(c3ccccc3)C2=O)c1', 'ClCc1cccc(I)c1O', 'ClCc1cccc(F)c1OC[C@@H]1CCCCC[C@H]1CO', 'ClCc1cccc([C@@H]2CCCN(CN3CCOCC3)C2)c1', 'ClCc1cccc(-c2nc3ccccc3n2Cc2ccccc2)c1', 'ClCc1cccc(Sc2nncn2C2CC2)c1', 'ClCc1cccc(Cl)n1', 'ClCc1cccc(Br)c1[N+](=O)[O-]', 'ClCc1csc2ncncc12', 'ClCc1csc2nc(C(F)(F)F)cn12', 'ClCc1csc2nc(N3CCC[C@@H](C(=O)N4CCC[C@H]4C)C3)nn12', 'ClCc1csc2nc([C@@H]3[C@@H](Cc4ccccc4)C(=O)N3)sc12', 'ClCc1csc2nc(-c3c[nH]c4ccccc34)oc12', 'ClCc1csc2nc(SCc3cccnc3)nnc12', 'ClC([O-])=NS(=O)(=O)c1c(F)cc(Cl)cc1Cl', 'ClCc1n[nH]cc1-c1cccc(Cl)c1', 'ClCc1n[nH]c2c(n1Cc1ccccc1)c(=O)n(CC(C(C)CO)[C@@H]2CNF)c1ccccc1', 'ClCc1n[nH]c2ccc(C(F)(F)F)cc12', 'ClCc1n[nH]c2c1CCC[C@H]([NH2+]CCc1ccccc1)C2', 'ClC[C@H](OCCO)C1CCN(C(=O)c2ccco2)CC1', 'ClC[C@H](O)COc1ccccc1', 'ClC[C@H](Br)Cc1ccccc1', 'ClC[C@H](Br)NC(=O)CC[C@H]1CCCO1', 'ClC[C@H](Br)c1n[nH]cc1C(F)(F)F', 'ClC[C@H](Br)[C@@]1(O)CCOC1', 'ClC([O-])CC[C@@H]1CCCC[C@@H]1n1ccnc1', 'ClC([O-])(Br)Cc1cccc(Cl)c1Cl', 'ClC([O-])=C1CCC(n2c(=O)[nH]c3ccccc32)CC1', 'ClC([O-])/C=C/c1ccc2nc(-c3cccc(F)c3)[nH]c2c1', 'ClC([O-])[C@@]1(O)Cc2ccccc21', 'COC[C@]1(C)CCCN(C(=O)C(=O)Nc2ccccc2)CC1', 'Oc1ccccc1/N=C/c1ccccc1COc1ccccc1Cl', 'Oc1ccccc1/N=C(\\NCC(=O)N1CCN(c2ccc(Cl)cc2)CC1)C(=O)[O-]', 'Oc1ccccc1/N=C1/C(=C/c2ccc3c(c2)OCO3)N=C2CCC[C@@H]21', 'Oc1cc(Cn2nc3ccccc3n2)ccc1Cl', 'Oc1cc(Cn2nc(Cl)ccc2=O)ccc1CO', 'Oc1cc(Cn2ncc(-c3cc([N+](=O)[O-])ccc3F)n2)ccc1Cl', 'Clc1cccc2c(-c3cc([N+](=O)[O-])cc4ccccc34)ccnc12', 'Clc1cccc2cc(CNc3cccnc3Cl)sc12', 'Clc1cccc2c1C[NH+](C[C@@H]1C(=O)NCC[NH+]1Cc1ccccc1Cl)[C@@H]1CC=C[C@H]21', 'Clc1ccccc1NC[C@@]1(O)Cc2ccccc2C(=O)N1c1ccccc1', 'Clc1ccccc1NS(=O)(=O)c1ccc(Cl)cc1', 'Clc1ccccc1Nc1nccn1/C=O', 'Clc1ccccc1N1CCN(Cc2cccc(S(=O)(=O)N3CCCCC3)c2)CC1', 'Clc1ccccc1-c1cc(C2CCOCC2)ncn1', 'Clc1ccccc1-c1cnn(-c2cc(F)cc3ccccc23)c1O', 'Clc1ccccc1-c1nc(CS(=O)(=O)N2CCCCC2)c[nH]1', 'Clc1cccc(/C=C/CCn2cccc(C(F)(F)F)c2=O)c1', 'Clc1cccc(/N=C2\\CCCCCc3nc[nH]c32)c1', 'Clc1ccc(-c2nonc2C2CC2)cc1', 'Clc1ccc(-c2nn(-c3ccccc3)cc2CN2CCN(CCc3cccc(F)c3)CC2)cc1', 'Clc1ccc(-c2nnn(-c3ccccc3)n2)cc1O', 'Clc1ccc(-c2csc3c(Cl)cccc23)cc1', 'Clc1ccc(-c2csc3sc4c(c23)CCC4)cc1', 'Clc1ccc(-c2csc3[nH]c4cc(Br)cnc4c32)cc1', 'Clc1ccc(-c2nonc2)cc1', 'Clc1ccc(CNC[C@@H]2CCCN2C(=O)c2ccc(Cl)cn2)cc1Cl', 'Clc1ccc(CNc2ccc(C(=O)NNC(c3ccccc3)nc2)C2CC2)cc1', 'Clc1ccc(CN2C(=O)CNC2=O)cc1', 'Clc1ccc(CN[C@@H]2CCC[NH2+][C@H]2c2ccc(F)cc2)cc1', 'Clc1ccc(CNS(=O)(=O)c2ccc(Br)cc2F)cc1', 'Clc1ccccc1[C@H](CCO)Nc1ccc2nccnc2c1', 'Clc1ccccc1[C@H](c1ccccc1)n1c([O-])nnc2ncnc2c1=O', 'Clc1ccccc1[C@H](O)CCCNC[C@H]1CCCC[NH+]1C', 'Clc1ccccc1[C@H](N[C@H]1CCSc2ccc(Cl)cc21)c1ccccc1', 'Clc1ccccc1[C@H](Cl)c1ccc(F)cc1', 'Clc1ccccc1[C@H](Br)C1(C[NH+]2CCCCCC2)CC1', 'Clc1ccccc1[C@H]([NH3+])c1cnn2c1CN(Cc1ccccc1Cl)C2', 'Clc1ccccc1[C@H]([NH2+]CCO[C@H]1CCS(=O)(=O)C1)c1nccs1', 'Clc1cc(/C=N\\NC2=N[C@H]3OCO[C@@H]32)ccc1Br', 'Clc1cc(/C=N/c2cccnc2)ccc1Cl', 'Clc1ccsc1S(=O)(=O)N1CCN(c2ccccc2)CC1', 'Clc1ccsc1F\n\n\n\n', 'Clc1ccsc1-c1cccc(C(F)(F)F)c1', 'Clc1ccsc1-c1nc([C@@H]2C[NH2+]CCO2)no1', 'ClCc1ccn(CCn2cccn2)c1', 'ClCc1ccn(C(=O)c2ccc(F)cc2Cl)c1', 'ClCc1ccn(Cc2nc(N3CCOCC3)nc[nH+]2)c1', 'ClCc1ccn(C2CCCC2)n1', 'ClCc1ccn(COc2ccccc2)n1', 'ClCc1ccn(C[C@@H]2CCCCN2S(=O)(=O)c2ccc(F)cc2)c1', 'ClCc1ccn(Cn2nnc(-c3cccc(Br)c3)n2)c1', 'ClCc1ccn(C[C@H](O)c2ccccc2)c1', 'ClCc1ccn(C[C@@](O)(c2ccccc2)N2CCOCC2)c1', 'ClCc1cc([N+](=O)[O-])ccc1Br', 'ClCc1cc(Cl)cc(C(=O)NCC[NH+]2CCOCC2)c1', 'ClCc1cc(Br)ccc1Oc1ccccc1F', 'ClCc1cc(F)ccc1-n1cncn1', 'ClCc1cc([N+](=O)[O-])ccc1OCc1ccccc1F', 'ClCc1cc([N+]([O-])=Nc2ccccc2Cl)oc1N1CCCC1', 'ClCc1cn(CCO)nn1', 'ClCc1cn(Cc2ccccc2)nc1-c1cccnc1', 'ClCc1cn(C2CCCCCC2)nn1', 'ClCc1cn(C[C@@H]2CCCO2)c(C(=O)[O-])c1', 'ClCc1cn(C[NH+]2CCCC[C@H]2c2ccccc2)nn1', 'ClC[C@@H]1CCC[NH+](Cc2csc(-c3cccnc3)n2)C1', 'ClC[C@@H]1CCc2cc(Oc3ccc(Br)cc3)oc21', 'ClC[C@@H]1CCO[C@]1c1c(F)cccc1F', 'ClC[C@@H]1CCN(Cc2ccc(F)cc2)C1', 'ClC[C@@H]1CC[C@@H](N[C@@H]2CCCC[C@H]2O)[C@H]1CO', 'ClC[C@@H]1CC[NH2+]C1', 'O=[N+]([O-])c1ccn([C@H](CCO)c2ccc(F)cc2)n1', 'O=[N+]([O-])c1ccn([C@H](OC[C@H](O)c2cccc(Cl)c2)[NH+]2CCCC2)c1', 'O=[N+]([O-])c1ccn([C@H](c2ccsc2)Cn2cc[nH+]c2)c1', 'CNc1[nH+]/c(=C\\c2ccccc2Cl)nc([O-])c(OC)c1OC', 'CNc1[nH+]ccc(C[NH+]2[C@@H](C)C[C@H]([NH2+]C)CC[C@@H]2C)n1', 'Nc1c(F)cccc1N/C=C(\\C#N)C(=O)Nc1ccc(Cl)cc1', 'Nc1c(Cl)cccc1S(=O)(=O)N[C@@H](c1ccccc1)c1cccc2nc[nH]c12', 'Nc1c(Cl)nc2ccccc2c1N[C@H]1Cc2ccccc2C1', 'Nc1c(F)cc(Cl)cc1CN1CC[C@@](Cc2ccccc2)C1', 'Nc1c(Oc2cccc(C(=O)N3CCOCC3)c2)cnn1N', 'Nc1c(F)c(F)cc(F)c1F', 'Nc1c(Cl)nsc1NCc1csc2ccccc12', 'Nc1c(Cl)n2ncnc2n1CCS(=O)(=O)CC(=O)N1CCCC1', 'Nc1c(Cl)ncn2ccnc12', 'Nc1c(/C=N\\N(CCCC#Cc2ccccc2)c2ccco2)cc2ccccc2n1', 'Nc1c(O[C@@H]2CCOC3(CCOCC3)C2)nc(N2CCOCC2)n1C1CC1', 'Nc1[nH]nc2nc(SCc3ccccc3C(F)(F)F)cnc12', 'Nc1[nH]nc2ncccc12', 'Nc1[nH]nc2ncn(Cc3ccccc3Cl)c12', 'Nc1[nH]nc(SCCOc2ccc(F)cc2)c1[N+](=O)[O-]', 'Nc1[nH]nc(Sc2ccccc2)n1', 'Nc1[nH]nc(S[C@@H]2CCOC2)c1[N+](=O)[O-]', 'Nc1[nH]nc(S[C@H](C(=O)[O-])c2ccccc2)c1-c1ccccc1', 'N[C@@H]1[C@H](c2cccc(F)c2)CCN(Cc2cncc(-c3cc(OC)c(OC)cc3)n2)C1', 'N[C@@H]1[C@H](OCc2ncc[nH]2)OC[C@@H](OCc2ccccc2)C1', 'N[C@@H]1[C@H](Cl)CCC[C@@H]([NH+]2C[C@@H](C)O[C@H](O)C2)C1', 'N[C@@H]1[C@H]([NH2+]C2CCN(c3ccc(C(F)(F)F)cn3)CC2)CCC[NH+]1C', 'Oc1cc(-n2cc[nH+]cc2=O)cc(I)c1', 'Oc1cc(-n2cc[nH+]c2)ccc1NC(=O)[C@@H]1C[C@H]2C=C[C@@H]1C2(Cl)Cl', 'Clc1nnnn1/N=C/C(=O)c1ccc(F)cc1', 'Clc1nnnn1-c1c(C(F)(F)F)cc(C(=O)[O-])c2ccccc12', 'Clc1nnnn1-c1c(F)cccc1F', 'Clc1nnnn1-c1c(-c2ccc(F)cc2)[nH]c2ccccc21', 'Clc1nnnn1-c1c(Cl)cccc1Cl', 'Clc1nnnn1-c1c(Br)ccc(Br)c1[O-]', 'Clc1nnnc(Br)c1I', 'Clc1cc(Cl)nnc1N/N=C/c1ccc(Cl)cc1', 'Clc1cc(Cl)nn1/N=C(/[O-])c1ccc2c(c1)OCO2', 'Clc1cc(Cl)nn1CCOC1CC1', 'Clc1ccsc1SCC(=O)N1CCC[C@H]1c1ccc2c(c1)OCO2', 'Clc1ccsc1SCc1ccc(C(=O)[O-])cc1', 'Clc1ccsc1SC1CCN(c2ncnc3[nH]c[nH+]c23)CC1', 'Clc1ccsc1SC[C@@H]1CN(Cc2ccc(Cl)cc2)CCO1', 'Clc1ccsc1SC[C@H]1Cc2ccccc2S1', 'Clc1ccsc1SC/C=C/c1nc2sccn2c1Cl', 'OCc1n[nH]c(NCc2ccccc2Cl)n1', 'OCc1n[nH]c(Nc2ccc(Br)cc2)n1', 'OCc1n[nH]c(N2CCOCC2)n1', 'OCc1n[nH]c(NNC(=O)N[C@@H](c2ccccc2)C2CCCCC2)n1', 'OCc1n[nH]c(N[C@H]2CC[S@](=O)C(Br)(Cl)C2)n1', 'OCc1n[nH]c(N/N=C/c2ncc(Br)cn2)n1', 'O=C([C@H](SC1=NC=NC1=O)c1ccccc1)c1ccc(Br)cc1', 'O=C([C@H](SC1CCCCC1)N1CCCC1)OC1CCCCC1', 'O=C([C@H](SC1NN(c2ccccc2)CC1)N1C[C@@H](CO)CC1)c1ccccc1', 'O=C([C@H](SC1[NH+]=c2ccccc2=[NH+]1)c1ccc(Cl)cc1Cl)NC1CC1', 'Clc1nnn(-c2cn(Cc3cccc(C(F)(F)F)c3)cn2)c1C#N', 'Clc1nnn(-c2cncc(C(=O)[O-])c2)c1Cl', 'Clc1nnn(-c2cnn(Cc3cccnc3)c2Cl)c1Cl', 'Clc1nnns1\n\n\n\n\n\n', 'NS(=O)(=O)/C=C/CSc1ccccc1', 'NS(=O)(=O)/C=C/c1cccnc1Cl', 'NS(=O)(=O)/C=c1/sc[n+]([O-])c2ccccc12', 'NS(=O)(=O)/C=c1/c(=N/C(=O)c2cc(Cc3ccccc3)n[nH]2)sc2c1CCCCCCC2', 'O=C(Cc1coc(-c2ccccc2Cl)n1)NC1COC1', 'O=C(Cc1coc2ccccc12)Nc1ccccc1-c1ccccc1', 'O=C(Cc1c[nH]c(C(=O)[O-])c1)Nc1cccc([N+](=O)[O-])c1', 'O=C(Cc1c[nH]cc1Cl)NCc1nc(C2CCCCC2)no1', 'O=C(Cc1c[nH]c2ccccc12)Nc1ccc(Br)cc1O', 'O=C(CSCc1ccccc1)Nc1ccc(S(=O)(=O)C(F)F)cc1', 'O=C(CSCNC(=O)c1ccc([N+](=O)[O-])cn1)Nc1cnc(-c2cccnc2)o1', 'O=C(CSCNc1ccccc1C(F)(F)F)NCc1ccccc1', 'O=C(CSCN1C(=O)/C(=C/c2ccccc2)SC1=S)NCc1ccc2c(c1)OCO2', 'O=C(CSCNOc1cccc(C(F)(F)F)c1)c1cccc([N+](=O)[O-])c1', 'O=C(CSCNS(=O)(=O)c1ccc(Cl)cc1)NNC(=O)OCc1ccccc1', 'O=C(Cc1n[nH]nc1C(F)(F)F)Nc1cnn(Cc2ccccc2)c1', 'O=C(Cc1n[nH]n1C1CC1)Nc1ccc2oc(-c3cccs3)nc2c1', 'O=C(Cc1n[nH]n2c[nH+]cc12)Nc1cccnc1', 'O=C(C1CC=CC1)N1CCC(OC[C@H]2C=N[C@H]3CS(=O)(=O)C[C@H]32)CC1', 'O=C(C1CC=[NH+]C1)N1CCc2sccc2-c2cc(Cl)ccc21', 'O=[N+]([O-])c1c([O-])c2sccn2c1C1CC1', 'O=[N+]([O-])c1c([O-])c2[nH]cnc2n1C[C@@H]1C[C@@H]2CC[C@@H]1C2', 'O=[N+]([O-])c1c([O-])c2c(c1-c1ccccc1)c([O-])c(=O)n2CCc1ccccc1', 'O=[N+]([O-])c1c([O-])c2nc(-c3ccccc3)n[nH]c2c1O', 'OC[C@@]1(c2ccccc2)Nc2sccc2N1', 'Oc1ccc(Oc2nc(=O)c3cccnc3n2N)c([N+](=O)[O-])c1Cl', 'Oc1ccc(Oc2nc(N3C[C@H](C)O[C@H](Sc4ccc(F)cc4)C3)no2)cc1', 'Oc1ccc(Oc2nc(-c3ccccc3)oc2-c2ccccc2)cc1Cl', 'Oc1ccc(Oc2nc(Br)c3sccc3n2)cc1', 'Oc1ccc(/C=C(\\Cl)CS(=O)(=O)c2cccc(Cl)c2)cc1', 'Oc1ccc(/C=C(/CO)C(F)(F)F)cc1', 'Clc1cc(F)cc(C(=O)N2Cc3ccccc3[C@@H](c3ccccc3)C2)c1', 'Clc1cc(F)ccc1[C@H](O)c1nncn1C1CC1', 'Clc1cc(F)cc2c1CCN(C(=O)Cc1ccc(OC(F)F)cc1)CC2', 'O=C(/N=C/c1ccc([N+](=O)[O-])cc1)NCCC(F)(F)F', 'O=C(/N=C/c1ccccc1F)N(Cc1ccccc1Cl)c1cccs1', 'O=C(/N=C/c1cc(-c2ccc(F)cc2)n(-c2ccccc2Cl)n1)NCC1(O)CCOCC1', 'O=C(/N=C/c1ccc(-n2cccn2)cc1)N1CCC(O)CC1', 'O=C(/N=C/c1cc2ccccc2[nH]1)C1(c2cccnc2)CC1', 'O=C(c1cn2ccsc2n1)N1CCC[C@H](CSc2ccncc2)C1', 'O[C@@H]([C@@H]1CC(C(C)C)=C(C)CS1)c1ccc(Cl)cc1', 'O=C(c1cccn1Cc1ccccc1)N1CCN(c2ccc[nH+]c2)CC1', 'O=C(c1cccn1c1ccc(Cl)cc1)N1CCOC[C@@H]1CO', 'O=C(c1cc(Cl)c(Cl)c(Cl)c1)N1CCN(c2cccc[nH+]2)CC1', 'O=C(c1cc(Cl)ccc1N1CC[NH+](C2CC2)CC1)N(Cc1ccsc1)C[C@@H]1CCCO1', 'O=C(c1cc(Cl)c2c(c1)OCCO2)N1CCN(c2ncccn2)CC1', 'O=C(c1cc(Cl)cnc1Cl)N1CCC(Oc2ccccc2)CC1', 'O=C(c1cc(Cl)c[nH]1)N1CCc2ccc(NN)cc21', 'O=C(c1ccco1)c1cc(S(=O)(=O)NC[C@H]2CCCO2)c(F)cc1F', 'O=C(c1ccco1)N1CCOc2cc(-c3ccccc3F)ccc21', 'O=C(c1ccco1)[C@@H]1CCCN1C(=O)CSc1ccc(F)cc1', 'O=C(c1ccco1)[C@H]1Sc2ccccc2NC(=O)[C@H]1Cc1cccc2ccc(Cl)cc12', 'O=C(c1cn[nH]c1NC1CCN(C(=O)COc2ccccc2)CC1)N1CCO[C@@H](C2CC2)C1', 'O=C(c1cn[nH]c1N1CCC[C@@H]1c1ccccc1)N1CCC[C@@H]1c1nc2ccccc2[nH]1', 'ClCc1cc(Br)c(Br)s1', 'ClCc1cc(Br)c2c(c1)OCO2', 'ClCc1cc(Br)cnc1N1CC[C@H]([NH+](C)Cc2ccccc2)C1', 'ClCc1cc(Br)c[nH]c1=O', 'O=C(CCc1ccccc1)Nc1ccc(-n2ncn(C)c2=O)cc1', 'O=C(CC1=CCCCC1)NNC(=O)[C@@H]1CC(=O)N(c2ccc(C#N)cc2)C1', 'O=C(CC1CCN(c2ncccn2)CC1)Nc1ccc(F)c(F)c1', 'O=C(CC1(C2=CC(=O)c3ccccc32)CC1)N1CCCC1', 'O=C(CC12CC(CC(CO)C1)CCC2)Nc1ccc(F)c(Cl)c1', 'Oc1ccccc1/C(Cl)Cc1ncccc1OC1CCCCC1', 'Oc1ccccc1-n1cccn1', 'Oc1ccccc1-n1cccc1C[NH2+]Cc1cnn(-c2ccc(F)cc2)c1', 'Oc1ccccc1-n1nnnc1SCC(=O)N1CCC(C2c3ccccc3C2)CC1', 'Oc1ccccc1O[C@@H](c1ccccc1)c1ncccc1O', 'Oc1ccccc1O[C@@H]1CCC[C@@H](C(=O)[O-])C1', 'Oc1cccc(Br)c1C[N+](=O)[O-]', 'Oc1cccc(Br)c1C(F)(F)F', 'Oc1cccc(Br)c1Cc1ccccc1', 'Oc1cccc(Br)c1C1CC[NH+](Cc2ccc(C#N)cc2F)CC1', 'Oc1cccc(Br)c1COc1cccc(Br)c1', 'Oc1cccc(Br)c1CNCc1ccc(Br)c(F)c1', 'Oc1cccc(Br)c1Cn1cccn1', 'Oc1cccc(Br)c1C[C@H]([NH2+]Cc1ccc(Cl)cc1)c1cccc(F)c1', 'Oc1cccc(Br)c1C[NH+]1CCC[C@@H]1c1cncn1C1CC1', 'Oc1cccc(Br)c1C[NH2+][C@@H]1CCCc2ccccc21', 'Oc1ccc([N+](=O)c2oc3c(c2[C@@H](NCc2ccccc2)C[C@H](c2ccc(F)cc2)N3)c2ccc(F)cc2)cn1', 'Oc1ccc([N+](=O)c2occc2Cl)c2c1OCCO2', 'Oc1ccc([N+](=O)c2ocnc2)c2[nH]ccc12', 'ClCc1noc(-c2ccco2)n1', 'ClCc1noc(C[NH2+]Cc2cnn(Cc3ccccc3)n2)n1', 'ClCc1noc([C@@H]2CCCCC[C@@H]2[NH2+][C@@H]2CCCC[C@@H]2O)n1', 'O=C(/C=C\\CC1CCCC1)NC[C@@](O)(c1cccs1)C1CC1', 'O=C(/C=C\\C(Cl)(Cl)Cl)[C@H]1CCS(=O)(=O)C1', 'O=C(/C=C\\Cc1ccc(Oc2ccccc2)c(F)c1)c1cc(O)ccc1F', 'O=C(/C=C\\C1(C(=O)Nc2cncc(-c3ccccc3)n2)CC1)N(C1CCCC1)S(C)c1cccnc1', 'O=C(/C=C\\C[N-]c1ccc(Cl)cc1Cl)C1CCCCC1', 'O=C(/C=C\\C=C\\c1ccccc1)Nc1ccc2c(c1)OCO2', 'O=C(/C=C\\COC1CC[NH+](Cc2cccs2)CC1)Nc1cccc2c1CCCC2', 'O=C(/C=C\\CN1CCSC1=O)c1ccc(Cl)c(Cl)c1', 'O=C(/C=C\\CS(=O)(=O)N1CCCSCC1)Nc1cc(Br)ccc1O', 'CCNc1ccc(C(=O)N[C@@H](C)c2ccccc2Br)cc1OC', 'CCN1C(=O)c2ccccc2[C@@H]1C(=O)Nc1ccc(Br)cc1', 'CCN1S(=O)C[C@@]2(C(=O)N3CCCCC3)CC(=O)N=C2N1', 'C=C[C@@H](NC(=O)c1ccc(F)c(C)n1)c1ccccc1Cl', 'C=C[C@@H](NS(=O)(=O)[C@H]1CCCN(c2ncc[nH]2)C1)c1ccccc1', 'Oc1cc(O[C@@H](CC(F)F)c2c[nH]nc2-c2ccccc2)ccc1O', 'Oc1cc(O[C@@H](C(=O)[O-])c2cc(=O)[nH]c3ccccc23)ccc1O', 'Oc1cc(O[C@@H](C)C(=O)N2CCC[C@H]2C[C@@H]2CCCO2)ccc1Cl', 'Oc1cc(O[C@@H](C2CCCC2)N2CCCC2=O)cc(Br)c1Cl', 'Oc1cc(O[C@@H](CO)c2ccccc2)ccc1-c1ccc(Cl)cc1', 'Oc1cc(O[C@@H](CN2CCOCC2)[C@@H]2CCC[NH+]2Cc2ccccc2)ccc1O', 'Oc1cc(O[C@@H](C#N)c2ccccc2)ccc1CN1CCOCC1', 'Oc1cc(O[C@@H](C[NH+]2CCCC2)c2ccccc2)c(Cn2cccn2)c2ccccc12', 'Clc1ccc(-c2nn(-c3ccc(F)cc3)c([O-])n2)cc1', 'Clc1ccc(-c2nn([C@@H]3CCO[C@H]2c2ccccc2F)C3)cc1', 'CCn1nc(C)c(CNC(=O)c2cccc(-n3cccc3)c2)c1C', 'CCn1nnc(CNC(=O)c2ccc(F)cc2Cl)n1', 'CCn1nnc(CN2CCN(C(=O)Cc3ccc(F)cc3F)CC2)n1', 'CCn1nnnc1[C@@H]1CCCCN1C(=O)CN(C)C(=O)c1cccnc1', 'CCS(=O)(=O)c1cc(C(=O)[C@@H](C#N)c2ccncc2)cc(OC)c1OC', 'CCS(=[NH2+])=c1/c(=N/C(=O)COC)sc2c(C)cc(C)cc12', 'O=[N+]([O-])c1c(Br)sc2[nH]cnc12', 'O=[N+]([O-])c1c(Br)sc2c1[C@H]1CC[C@@H]2C[C@@H](O)[C@@H]1N1c2ccccc2C1=O', 'O=[N+]([O-])c1c(Br)sc2ncn(Cc3cccnc3C(F)F)c12', 'N=Cc1c(OCCSc2ccc(F)cc2)nc2ccccn12', 'N=Cc1c(NC(=O)[C@]2(c3ccc(F)cc3F)NC(=O)C(=O)N2C)cccc1OC(F)F', 'N=Cc1c(F)cccc1OCCOC(F)F', 'N=Cc1c(-c2ccccc2)nnnc1Sc1ncnc(N2CCC(C)CC2)c1CC', 'Nc1[nH]nc2ncc(-c3ccc(-c4ccccc4)cc3)nc12', 'Nc1[nH]nc2ncccc12', 'Nc1[nH]nc2c1C[NH+](CC(=O)N(C1CC1)[C@H]1CCS(=O)(=O)C1)CCC2', 'Nc1[nH]nc2c1[C@@H](c1ccc(Cl)cc1)C1=C(CCCC1=O)C2', 'Nc1[nH]nc2c1-c1nc(N)ccc1N2', 'Nc1[nH]nc2c1SC/C2=C(\\N)Nc1ccccc1', 'Nc1[nH]nc2c1/C(=C(/[O-])Nc1ccc([N+](=O)[O-])cc1)S2', 'COc1nc2nc(CCc3ccc(F)cc3)nn2c(=O)n1Cc1ccccc1', 'COc1nc2ncc(C)cc2s1', 'COc1nc(C(=O)N2CC[C@]3(CCCC[C@H]3C)C2)no1', 'COc1nc(C)cc(C)c1NC(=O)C(=O)Nc1ccc2[nH]c(=O)[nH]c2c1', 'COc1nc(C2(NC(=O)c3cc(C(C)C)no3)CCCC2)cs1', 'COc1nc(CN(C)c2nc(C)nc3c2c(C)nn3-c2cccc(C)c2C)cs1', 'COc1nc(C[NH+]2CCN([C@@H](C)CC(F)(F)F)CC2)c2ccccc2n1', 'O[C@@H](Sc1nnc(-c2ccccc2)o1)C(=O)Nc1ccccc1', 'O[C@@H](Sc1nnns1)C(=O)Nc1ccc(Cl)cc1', 'O[C@@H](Sc1cc2c(cc1[N+](=O)[O-])CC[NH2+]C2)C(=O)[O-]', 'O[C@@H](Sc1ccccc1)C(=O)N(Cc1ccc(Cl)cc1)C1CC1', 'O[C@@H](Sc1ccncc1)c1cccc(Br)c1', 'Cc1noc2nc(CN3CCO[C@@H](OCc4ccc(F)cc4)C3)c(C#N)cc12', 'C[C@@H](OCC[NH3+])[C@H]1C[NH2+]CC[C@@H]1c1c(F)cccc1F', 'C[C@@H](OC(=O)c1ccc(OCC#N)cc1)C(F)(F)F', 'C[C@@H](OC[C@H]1CCCCO1)C(=O)Nc1ccc([N+](=O)[O-])cc1', 'C[C@@H](OC[C@@H]1CCCO1)C(=O)OCC(=O)N1CCN(C)CC1', 'O[C@@H]([C@@H]1CCOc2ccccc21)[C@@H]1CCOC2(CCOCC2)C1', 'O[C@@H]([C@@H]1Cc2ccccc2O1)C1(c2ccccc2)CCOCC1', 'O[C@@H]([C@@H]1C=c2ccccc2=[NH+]1)c1cc([N+](=O)[O-])ccn1', 'O[C@@H]([C@@H]1COc2ccccc21)c1cc(F)ccc1F', 'O[C@@H]([C@@H]1CSc2ccc(F)cc21)c1ccc(Br)cc1', 'O[C@@H]([C@@H]1C[C@H]1c1ccc(F)cc1)c1ccc(F)cn1', 'N=CCc1ccc(Cl)c(CCC(=O)Nc2ccc(F)cc2)c1', 'N=CC1=C[C@@H](Sc2nnnn2C2CCC(C(N)=O)CC2)C(=O)N1c1ccccc1', 'N=CC1CCN([C@H](C(=O)N2CCCC2)c2ccccc2)CC1', 'N=CC1(Sc2nc(Cl)ncn2)CCCCC1', 'ClC[C@H]1CCCN(c2ccccc2)C1', 'ClC[C@H]1CCc2ccccc21', 'ClC[C@H]1CCOC2(CCCCC2)C1', 'ClC[C@H]1CC[C@@H]([NH+](CC(=O)[O-])CO)C1', 'ClC[C@H]1CCS[C@@H]1c1cccc(Cl)c1', 'ClC[C@H]1CC[C@H](Cc2ccc(F)cc2)[N+]12CCCCCC2', 'Oc1ccccc1[C@H]1N=NC[C@@H]1c1nc(-c2ccc(Cl)cc2)no1', 'Oc1ccccc1[C@H]1CC[NH2+][C@@H]2[C@H]1c1ccc(F)cc12', 'Oc1ccccc1[C@H]1OCC[C@@H]1F', 'C=C(Br)CN(C(=O)c1cccn1CC)[C@@]1(C)CCCO1', 'NS(=O)(=O)Cc1noc(CC[S@@](=O)Cc2ccccc2)n1', 'NS(=O)(=O)Cc1ccsc1', 'NS(=O)(=O)Cc1cc(NCC[NH+]2CCCCC2)ccn1', 'NS(=O)(=O)Cc1ccc(Cl)cc1', 'N=C1\\[C@@H](SCCn2c(N)nc3ccccc32)[C@H]2C=CCCCN12', 'O=C(c1cc([N+](=O)[O-])c2ccccc2n1)N1CCCN(c2nnc(-c3ccc(F)cc3)o2)CC1', 'Cc1nonc1CCCN(C)C(=O)CCNc1ccccc1', 'Cc1nonc1C(=O)Nc1cc(C(=O)NCCC#N)ccc1Br', 'Cc1nonc1Cc1nc(C)c(C)s1', 'Cc1nonc1CN(Cc1ccccc1)c1ncccn1', 'Cc1nonc1C[C@H](CS)Cc1ccccc1', 'Cc1nonc1C[NH+]1CCC(NC(=O)c2ccc(C[NH3+])cc2)CC1', 'Cc1sc(C#N)c(S[C@@H](C(=O)[O-])c2ccc(Cl)cc2)n1', 'Cc1sc(C(=O)N/N=C/c2ccc(C(C)C)cc2)c(C#N)c1C', 'Cc1sc(C[NH2+][C@@H]2CCCC[C@@H]2CO)c(C)c1C', 'O=C(CNc1cn(C2CCCCC2)c[nH+]c1=O)Nc1cccc(Cl)c1Cl', 'O=C(CNc1cnc2ccccc2n1)NCCn1cc[nH+]c1', 'O=C(CNc1cnn(Cc2ccccc2)c(=O)c1Cl)Oc1ccc([N+](=O)[O-])cc1', 'O=C(CS(=O)(=O)[C@@H](C(=O)[O-])C(F)F)N[C@@H]1CCCC[C@@H]1O', 'O=C(CS(=O)(=O)[C@@H]1CCSC1)Nc1cc(C(F)(F)F)cs1', 'O=C(CS(=O)(=O)[C@@H]2CCS(=O)(=O)C2)N1Cc2ccccc2C1', 'COC(=O)C1(C#N)CC[C@H](n2ccc(=O)[nH]c2=O)CC1', 'COC(=O)c1cnn(Sc2nncn2C2CC2)c1C', 'COC(=O)Nc1ccc(O[C@@H](C(C)C)[NH+](C)C)cc1', 'COC(=O)[C@@H](C)NS(=O)(=O)c1ccccc1Cl', 'COC(=O)[C@H](C[NH+]1CCCCC1)c1ccc(F)cc1', 'COC(=O)[C@]1(F)CCN(C(=O)c2cc(F)c(F)cc2Br)C1', 'COC(=O)[C@@]1(F)CC[NH+](CC(=O)[O-])C1', 'Oc1cc(F)nc(SCC(=O)Nc2ccc(S(=O)(=O)C(F)F)cc2)c1', 'Oc1cc(F)nc(Sc2cc(Br)ccc2O)n1', 'Oc1cc(F)nc(S[C@@H]2CCOC3(CCOCC3)C2)c1', 'Oc1cc(F)nc(S[C@H]2CC[NH2+]CC2)c1N1CCCC1', 'Oc1cc(F)nc(S/C=C(c2ccccc2)c2ccc(Cl)cc2)c1', 'NS(=O)(=O)CC(=O)Nc1ccc(C(=O)NCc2ccco2)cc1', 'NS(=O)(=O)CCc1nc(-c2ccccc2)cs1', 'NS(=O)(=O)CCNC(=O)c1cccc(NC(=O)c2ccco2)c1', 'NS(=O)(=O)CCN(C(=O)C1CC1)C1CCC(c2ccccc2)CC1', 'NS(=O)(=O)CCNc1ccc(Oc2cccc(C#N)c2)cc1', 'NS(=O)(=O)CCNS(=O)(=O)c1ccccc1', 'NS(=O)(=O)CCCC1CC[NH+](CCCC2=CCCCC2)CC1', 'NS(=O)(=O)CCCc1cccc(C(=O)N(C2CC2)C2CCCCC2)c1', 'NS(=O)(=O)CCCNCc1c(Cl)cccc1Cl', 'NS(=O)(=O)CCCSc1nc2cc(Cl)ccc2c(=O)[nH]1', 'NS(=O)(=O)CCC[NH+](C(=O)c1ccc(Cl)s1)c1ccccc1', 'NS(=O)(=O)N[C@@H]1CCN(c2ccccc2F)C1', 'NS(=O)(=O)N[C@@H]1c2ccccc2C(=O)[C@@]12CCC[C@@H]2[NH+]C(=O)c1ccccc1Br', 'NS(=O)(=O)Cc1nnc(-c2ccccc2)n1-c1ccc(Cl)c(Cl)c1', 'NS(=O)(=O)Cc1nnn(-c2ccccc2)c1N', 'NS(=O)(=O)Cc1ncsc1CO', 'NS(=O)(=O)Cc1nc(Cc2ccccc2)cs1', 'NS(=O)(=O)Cc1ncco1', 'NS(=O)(=O)Cc1nc2cccc(Cl)c2[nH]1', 'NS(=O)(=O)C[C@H](Cc1c[nH]c2ccccc12)NC(=O)c1ccccc1[N+](=O)[O-]', 'NS(=O)(=O)C[C@H](c1cccs1)n1nnc2cc(Cl)ccc21', 'NS(=O)(=O)C[C@H](O)c1ccc(Cl)c(Cl)c1', 'NS(=O)(=O)C[C@H](NC(=O)Cc1csc(Nc2ccc(F)cc2)n1)c1ccccc1', 'NS(=O)(=O)C[C@H](SCc1ncoc1C1CC1)c1ccccc1', 'NS(=O)(=O)C[C@H](Br)C(=O)[O-]', 'NS(=O)(=O)C[C@H]([NH3+])c1cccs1', 'NS(=O)(=O)C[C@H]([NH2+]Cc1ccccn1)N1CC[NH+](C2CC2)CC1', 'Nc1nnc(Nc2cccc(Oc3ccccc3)c2)s1', 'Nc1nnc(-c2cccc(F)c2)cc1CN1CCC[C@@H](c2nc3ccccc3[nH]2)C1', 'Nc1nnc(SCC(F)(F)F)n1N', 'Nc1nnc(Cl)n1C1CCCCC1', 'Nc1nnnn1CC(=O)NCc1ccccc1OC(F)F', 'Nc1nnnn2c(C(=O)[O-])ccc12', 'Nc1nonc1-c1ccccc1', 'Nc1nonc1C(=O)C[C@H](O)c1ccccc1', 'Nc1nonc1Cc1ccc[nH]1', 'Nc1nonc1C=O', 'Nc1nonc1COc1cccc(-c2ccco2)n1', 'Nc1nonc1CN1CCN(C(=O)/C=C/C2CC2)CC1', 'Nc1nonc1Cn1ncc2[nH]c3ccccc3c21', 'Nc1nonc1C[NH+]1CC[C@@H](OS(=O)(=O)N(CCCCO)c2cccs2)C1', 'Nc1nonc1C[NH2+]C1CCN(CC(=O)c2ccccc2)CC1', 'Nc1c(F)c([N+](=O)[O-])ccc1-n1cccc1', 'Nc1c(F)c(O)c2ccccc2c1OC(=O)CSc1nccs1', 'Nc1c(F)c(N2CCCC2)c([N+](=O)[O-])cc1O', 'Nc1c(F)c(F)c(F)c(F)c1F', 'Nc1c(F)c(Br)ccc1F', 'Nc1cnnn1-c1ccccc1/C=C/C(=O)c1ccc(Br)cc1', 'Nc1cnnn1/C(Cl)=N/C=C/C(=O)[O-]', 'Nc1cnnn1-c1ccc(F)cc1C(=O)N[C@H](c1ccccc1)C1CC1', 'Nc1cnnn1Sc1ccccc1', 'Nc1nncn1CCN1C[C@@H](C)OCC1(C)C', 'Nc1nncn1c1ccc(Cl)cc1', 'Nc1nncn1[C@@H]1CC(=O)N(c2cccc(Cl)c2)C1', 'Nc1nncn1-c1ccccc1', 'Nc1nncn1/N=N/N=C/c1ccc([N+](=O)[O-])cc1', 'N=Cc1nonc1-c1ccc(OC(F)F)c(C)c1', 'N=Cc1nonc1Cc1cc(C)ccc1OCC(F)(F)F', 'N=Cc1nonc1[N-]S(=O)(=O)c1ccc2c(c1)OCCO2', 'N=Cc1nonc1N', 'N=Cc1nonc1S(=O)(=O)Cc1cccc([N+](=O)[O-])c1', 'N=Cc1nonc1[C@H]1NC(=S)N=C1N1CCN(c2ccccc2)CC1', 'Nc1c(CSCc2ccccn2)cnn1C', 'Nc1c(CSc2nnc(-c3ccccc3)cn2)oc2c1CN(Cc1ccc(F)cc1)CC2', 'Nc1c(/N=N/NC(=O)c2ccnc(OC3CCC3)c2)[nH]c2c1CCC[C@H]2O', 'Nc1c(Oc2[nH+]cccc2F)[nH]c2c1-c1ccc(Br)cc1NC2', 'Nc1c(Oc2cc(Cl)ncn2)cnn1-c1ccc(F)cc1', 'Nc1c(Oc2nc(N)nc3ccccc23)cccc1I', 'NS(=O)(=O)Nc1cc(C(=O)N2CCc3ccccc3C2)ccc1O', 'NS(=O)(=O)Nc1cc([S@@](=O)Cn2cccn2)ccc1C(F)(F)F', 'NS(=O)(=O)Nc1cc(NC(=O)c2ccc3c(c2)CCC3)ccc1Cl', 'NS(=O)(=O)Nc1cc(F)ccc1C(=O)NC[C@H](c1ccco1)[NH+]1CCCC1', 'NS(=O)(=O)Nc1cc([C@@H]2CCC[NH+]2CC2CC2)ccc1Cl', 'NS(=O)(=O)Nc1cc(-c2ccco2)nc2ccc(Cl)cc12', 'NS(=O)(=O)Nc1cc(S(=O)(=O)C=O)ccc1C(F)(F)F', 'NS(=O)(=O)Nc1cc(Cl)ccc1C(=O)NC[C@@H]1CCCO1', 'NS(=O)(=O)Nc1cc(Br)ccc1C(F)(F)F', 'Nc1[nH]ncc1-c1ccc([N+](=O)[O-])cc1', 'Nc1[nH]ncc1SCC(=O)N1C[C@@H](O)C[C@H]1c1ccc(Cl)cc1', 'Nc1[nH]ncc1S(=O)(=O)N1C[C@H]2CC=CC[C@@H]2C1', 'Nc1[nH]ncc1Sc1ccccc1', 'Nc1n[nH]nc1CNC(=O)C(=O)NC1CC1', 'Nc1n[nH]nc1CN(Cc1ccco1)[C@@H]1CCS(=O)(=O)C1', 'NS(=O)(=O)/N=c1\\sc(-c2ccc(F)cc2)cc(-c2ccco2)n1', 'NS(=O)(=O)/C(=N\\[N+](=O)[O-])C(=O)N1c2ccccc2S[C@H]1C(C)C', 'NS(=O)(=O)/C(=N/O)c1cccc([N+](=O)[O-])c1', 'N[C@@H](Sc1ccc(OCc2ccccc2F)cc1)c1ccn(C(F)F)n1', 'N[C@@H](c1cccs1)CO[C@@H]1C[C@H]1C', 'N[C@@H](c1ncccc1F)[C@@H]1Cn2cnnc2C1', 'NS(=O)(=O)/N=c1\\oc2c(cnc3ccccc13)C(=O)CCC2', 'NS(=O)(=O)/N=c1\\[nH]c(-c2cccs2)nc2cc(C(=O)[O-])ccc12', 'O=C([C@H](c1c(F)cccc1Cl)[NH+]1CCN(c2cccc(Cl)c2Cl)CC1)C1CCCC1', 'O=C([C@H](c1c(Cl)cccc1Cl)C(F)F)N1CCN(Cc2ccco2)CC1', 'O=C([C@H](c1c([N+](=O)[O-])nnc2sccc12)[NH+]1CCCC1)N1CCN(c2ncccn2)CC1', 'O=C(c1c[nH]c(-c2ccsc2)n1)N1CCC[NH2+]CC1', 'Clc1cc(/N=C/C(=O)N[NH+]2CCOCC2)cs1', 'Clc1cc(/N=C/c2cccc([N+](=O)[O-])c2)sc1O', 'OC[C@@]1(OC2CCCCC2)C[C@H]1c1cnc[nH]1', 'OC[C@@]1(O)CCc2ccccc2CN1c1ccc(F)cc1', 'OC[C@@]1(Oc2ccccc2-c2ccccc2)COc2ccccc21', 'O=[N+]([O-])c1cc(Cl)c(N2CCc3ccccc3N2)cc1[N+](=O)[O-]', 'O=[N+]([O-])c1cc(Cl)ccc1-c1nc(C(F)(F)F)cs1', 'O=[N+]([O-])c1cc(Cl)c2nc(N)sc2c1', 'O=[N+]([O-])c1cc(Br)c(O)c(F)c1C(F)(F)F', 'O=[N+]([O-])c1cc(Br)c2ccccc2c1OCc1ccccc1F', 'O=[N+]([O-])c1cc(Br)ccc1Oc1ccc(Cl)cc1', 'O=[N+]([O-])c1cc(F)ccc1N1CCC[C@H]1c1ccco1', 'O=[N+]([O-])c1cc(Nc2ccc(-n3cccn3)cc2)cc(S(=O)(=O)N2CCCC2)c1', 'O=[N+]([O-])c1cc(N[C@@H](CO)c2ccc(F)c(F)c2)cn1C1CCCC1', 'O=[N+]([O-])c1cc(N[C@@H]2CCN(c3ccc(F)cc3F)N2)cs1', 'O=[N+]([O-])c1cc(F)ccc1N1CCC([NH+]2CCCc3cc(F)ccc32)Cc2ccccc21', 'Oc1cccc2ncn([C@@H]3CCCC[C@@H]3O)c12', 'Oc1cccc2ncn(/N=C/c3c(Cl)cccc3F)c12', 'Clc1ccc(-c2cc([N+](=O)[O-])ncc2N2CCCC2)cc1Cl', 'Clc1ccc(-c2cc(=O)n3nc(-c4ccco4)s3c2)cc1', 'Clc1ccc(-c2cc(Oc3ccccc3)nc(SC3CC3)n2)cc1', 'Clc1ccc(-c2cc(NCc3ccco3)ccc2C[NH+]2CCCC2)cc1', 'Clc1ccc(-c2cc(F)cc(Br)c2)cc1F', 'Clc1ccc(-c2cc(-c3csc4ccccc34)[nH]n2)cc1', 'Clc1ccc(-c2cc(S(=O)(=O)N3CCN(c4ccccc4)CC3)co2)cc1', 'Clc1ccc(-c2cc(Cl)n[nH]2)cc1CO', 'Clc1ccc(-c2cc([C@H](C#N)C3CC3)cs2)cc1', 'Clc1ccc(-c2cc(Br)ccn2)cc1', 'Clc1ccc(-c2cc(/C=N/c3ccccc3)[nH]n2)o1', 'Clc1ccsc1F\n\n\n\n\n', 'C=C[C@@H](OC(=O)Cc1cccc(Cl)c1)c1ccccc1', 'C=C[C@@H](O)[C@H](C)c1cccs1', 'C=C[C@@H](Oc1ccccc1OC)c1ccc(F)cc1', 'C=C[C@@H](SC)c1nc(-c2cc(F)ccc2F)n[nH]1', 'C=C[C@@H](S(=O)(=O)c1ccc(C)cc1)c1nc(C)cc(C)c1[N+](=O)[O-]', 'O=[N+]([O-])c1cc(Cl)n(-c2ccc(F)cc2)n1', 'O=[N+]([O-])c1cc(Cl)ncc1[C@@H]1CCOC2(CCCCC2)C1', 'Clc1nnnn1/C=C/C(F)(F)F', 'Clc1nnnn1/C([O-])=N/c1ccc([N+](=O)[O-])cc1', 'CCCCCNC(=O)c1cc(NC(=O)C=C2CCCCC2)co1', 'CCCC(CCC(=O)[O-])C(=O)N1CCC(C)CC1', 'CCCCc1ccc(CNC(=O)NC(C)(C)c2cccc(C)c2)s1', 'CCCC1CC[NH+]([C@H](C)c2nnnn2CC)CC1', 'CCCCO[C@H]1C[C@H](O)CN1Cc1cccc(OC)c1', 'CCCCN(Cc1cccc(C#N)c1)C(=O)N[C@@H]1C[C@@H]1C', 'CCCC[C@@H](CC)CNC(=O)N[C@H]1c2ccccc2C[C@H]1O', 'CCCCn1ncc2c1N=C(C)O[C@@H]2c1ccccc1C', 'CCCCSC[C@@H](C)CNC(=O)c1cccc(Br)c1', 'CCCC[C@H](CC)c1ccc([N+](=O)[O-])cc1', 'CCCC[NH+](C)CC1CCN(c2nccnc2C#N)CC1', 'O=C([C@H](Cl)Br)C(=O)[O-]', 'O=C([C@H](Cl)Br)c1cccs1', 'O=C([C@H](Cl)Br)[C@@H](C)c1ccc(Cl)cc1Cl', 'O=C([C@H](Cl)Br)S(=O)c1ccc2ccccc2c1', 'O=C([C@H](Cl)Br)[C@H]1CN2CCC12', 'C=C([O-])/C=C(\\C)NC(C)(C)C', 'ClC(/C=N/c1cccc(Br)c1)c1ccccc1', 'ClC(/C=C(/C#N)c1ccccc1)Nc1ccccc1', 'ClC(/C=[NH+]/CC[NH+]1CCCc2ccc(C(F)(F)F)cc21)c1ccc(Cl)cc1', 'Nc1ncc([C@@H](C)[NH2+]C2CC2)s1', 'Nc1ncccc1[S@@](=O)Cc1nc[nH]n1', 'Nc1ncc2c(c1O)C[NH+](Cc1cccc(C(F)(F)F)c1)CC2', 'Nc1nccnc1SCC(=O)N[C@@H]1CCS(=O)(=O)C1', 'Nc1nccs1', 'C[C@@H](NS(=O)(=O)c1ccccc1F)C(=O)NCc1ccc(F)cc1', 'C[C@@H](NC(=O)CCl)[C@H]1CCc2c([nH]c3ccccc21)c1ccsc13', 'C[C@@H](NC(N)=[NH2+])[C@H]1CCOC2(CCCCC2)C1', 'Oc1cc(Br)cs1\n\n\n', 'Oc1cc(Br)cnc1S[C@@H]1C[C@@H](Br)CO1', 'Clc1nnnc(Cl)c1I', 'Clc1nnnc(Cl)n1', 'O=[N+]([O-])c1ccn(Cc2cccc(C(=O)N3CCCC3)c2)c1', 'O=[N+]([O-])c1ccn(Cn2c(=O)c3ccccc3n2Cc2ccccc2)c1', 'O=[N+]([O-])c1ccn(CCC[NH+]2CCCC[C@@H]2C2CCCCC2)c1', 'O=[N+]([O-])c1ccn(CC(=O)N2CCSc3ccccc32)c1', 'O=[N+]([O-])c1ccn(CCc2ccccc2Cl)c1', 'O=[N+]([O-])c1ccn(CC2=CS(=O)(=O)N(c3ccccc3)c3ccccc32)c1', 'O=[N+]([O-])c1ccn(CCNC(=O)Nc2ncn[nH]2)c1', 'O=[N+]([O-])c1ccn(CC[C@@H]2CCCC[NH2+]2)c1', 'O=[N+]([O-])c1ccn(CC[C@H]2CCC[C@@H]2O)c1', 'O=[N+]([O-])c1ccn(CC[NH+]2CCC[C@@H]2c2ccc(O)cc2)c1', 'Clc1cc(CCC(=O)N(Cc2cccs2)C[C@H](O)c2ccccc2)ccc1F', 'Clc1cc(CC(=O)N2CCC[C@@H]2c2ccco2)ccc1F', 'Clc1cc(CCc2c[nH]c3ccccc23)[nH]c1', 'Clc1cc(CC2(C[NH2+]CC[C@H]3CCCCO3)CCC2)ncn1', 'Clc1cc(CCOc2ccccc2)sc1NCc1ccccc1F', 'Clc1cc(CCN[C@H]2CCCc3ccccc32)ccn1', 'Clc1cc(CC[NH+]2CCN([C@@H]3CCOC3)CC2)ccc1Cl', 'Clc1ccc(-c2n[nH]c(NC[C@H]3CCCC[NH2+]3)n2)cc1', 'Oc1ccc(O[C@@H](C(=O)[O-])c2cccc(Cl)c2)cc1', 'Oc1ccc(O[C@@H](Cc2nc3cc(Cl)ccn3c2C(F)(F)F)C2CCCC2)cc1', 'Oc1ccc(O[C@@H](C2CCCC2)C(F)(F)F)cc1', 'Oc1ccc(O[C@@H](CO)C(=O)NCc2ccccc2F)cc1', 'Oc1ccc(O[C@@H](CNC(=O)C[C@H]2COCCO2)c2ccco2)cc1Cl', 'Oc1ccc(O[C@@H](Cn2cccn2)C(=O)c2c[nH]c3ccccc23)cc1F', 'Oc1ccc(O[C@@H](C[NH+](C)C)Sc2ccccc2)c(Cl)c1', 'Oc1ccc(/C=C/NC(=S)Nc2c(F)cccc2Cl)c2ccccc12', 'Oc1ccc(/C=C/N(CC(F)F)c2ccccc2F)cc1', 'Oc1ccc(/C=C/Nc2cccnc2)cc1', 'Oc1ccc(/C=C/N2CCOCC2)cc1', 'Oc1ccc(/C=C/N=C(\\[O-])COc2ccccc2)cc1', 'Oc1ccc(/C=C(\\Cn2c3ccccc3c3ccccc32)c2ccccc2Cl)cc1', 'Oc1ccc(/C=C(\\c2ccccc2)C(=O)Nc2cc(C(F)(F)F)c(Cl)c(Cl)c2)cc1', 'Oc1ccc(/C=C(\\Nc2ccccc2)C(=O)Nc2ccccc2)cc1', 'Oc1ccc(/C=C(\\Cl)/C=C(Nc2ccc(Cl)cc2)N2CCOCC2)cc1', 'Oc1ccc(/C=C(\\[O-])/C([O-])=N\\c2ncccn2)c(O)c1', 'Oc1ccc(/C=C2/C(=O)N(c3ccccc3)C(=O)N(c3ccc(Cl)cc3)c3ccc(F)cc32)cc1', 'Oc1ccc(/C=C2\\CCS(=O)(=O)C2)c(Br)c1', 'Oc1ccc(O[C@H](CC(F)F)c2cccc(Cl)c2)c(F)c1', 'Oc1ccc(O[C@H](C(=O)NC2CCCCCC2)C(F)(F)F)cc1', 'Oc1ccc(O[C@H](Cc2ccc[nH]c2=O)[C@@H]2CC[NH2+]C2)cc1', 'Oc1ccc(O[C@H](C2CC2)c2ccc(F)cc2F)cc1', 'Oc1ccc(O[C@H](CO)C(F)(F)F)cc1', 'Oc1ccc(O[C@H](CNC(=O)c2ccco2)N2CCSCC2)cc1', 'Oc1ccc(O[C@H](C[C@@H]2CCCCO2)C(=O)c2ccccc2)c2c1C(=O)N=C(c1ccccc1)[NH+]2', 'Oc1ccc(O[C@H](CSc2ccccc2)C(=O)NC2CC2)cc1', 'Oc1ccc(O[C@H](C[NH+]2CCCC2)c2ccccc2)cc1', 'O=C(c1cnccc1)[C@H]1C[C@@H](O)CO1', 'Oc1cc(C[NH2+][C@@H]2CCc3ccccc32)ccc1Br', 'ClCc1csc([N-]Sc2ccccc2)n1', 'OCc1nnn(CC2CC2)c1Br', 'OCc1nnnn1[C@H]1CCCN(c2cc(F)cc(F)c2F)C1=O', 'OCc1nn(SC[C@@H]2CCCO2)c2ccccc12', 'OCc1nn(Sc2ccc(Cl)cc2Cl)c2ccccc12', 'O=C(Cc1cc2occc2[nH]1)NNC(=O)c1cc(Cl)ccc1Cl', 'O=C(Cc1cc2c(cc(Cl)cc1[N-]S(=O)(=O)c1ccccc1)n2Cc1ccccc1)NCc1ccccc1', 'O=C(Cc1cc2n(n1)COCCO2)[C@H]1Cc2ccc(F)cc2O1', 'Clc1nncc(-c2cccc(F)c2)n1', 'Clc1nncc(Cl)c1N1CCN(C(=O)[C@@H]2C=c3ccccc3=[NH+]2)CC1', 'Clc1nncc(Cl)n1', 'Clc1cc(Cl)nc(COc2ccccc2F)n1', 'Clc1cc(Cl)nc([N-]S(=S)(=O)c2ccc(F)cc2)n1', 'Clc1cc(Cl)nc(Nc2ccc(F)cc2)c1', 'Clc1cc(Cl)nc(-c2cccnc2)n1', 'Clc1cc(Cl)nc(SCc2noc(C3CC3)n2)n1', 'Clc1cc(Cl)nc(Cl)c1Cl', 'Clc1cc(Cl)nc([O-])c1F', 'Clc1cc(Cl)nc([C@H]2CCC[NH2+]2)n1', 'ClCc1csc(-c2cc(CN(Cc3ccccc3)c3ccccc3)oc2C2CC2)c1Cl', 'ClCc1csc(-c2ccccc2)n1', 'ClCc1csc(-c2ncc(Br)s2)n1', 'ClCc1csc([N-]S(=O)(=O)c2cccc(F)c2)n1', 'ClCc1csc([N-]S[C@H]2C=CCC2)n1', 'OC[C@H]1OCC[C@H]2CCCC[C@@H]2C1', 'OC[C@H]1ON=C2CCCN2/C1=C/c1ccnc([N+](=O)[O-])c1', 'OC[C@H]1O[C@H](O)CS[C@H](OCc2ccccc2)C1', 'Clc1nnn(-c2ncn(-c3ccccc3)n2)c1-c1ccccc1', 'Clc1nnn(-c2ncn([C@@H](C)CCN3CCOCC3)n2)c1', 'Clc1nnn(C(F)F)c1C(=O)Nc1ccc2c(c1)N(C)c1ccc(Cl)cc1C(=O)N2', 'Cc1c(Br)cc2c(c1)N(C(=O)[C@@H](C)CCS(C)(=O)=O)CC2', 'Cc1c(-c2cccs2)ncnc1N1CCC([C@H]2CCN(C)C2=O)CC1', 'Cc1c(-n2cccc2)c2ccccc2n1CC(=O)N[C@H]1CCC[C@@H]1[S@@](=O)Cc1ccccc1', 'Cc1ncsc1C(=O)Nc1ccccn1', 'Cc1ncsc1CN(C)c1ccc(C(=O)N2CCCC2)cc1', 'Cc1ncsc1C(=O)N1CC=C(c2ccc(Cl)cc2Cl)CC1', 'Cc1ncsc1[C@@H](O)N1CCN(Cc2cccc(F)c2)CC1', 'Cc1ncsc1CCN(C)C(=O)CCNC(=O)OC(C)(C)C', 'Cc1ncsc1C(=O)N1CCN(C[C@H](O)C(C)(C)CCC#N)CC1', 'Cc1ncsc1CO[C@@H]1CCC[C@@H](C)C1', 'Cc1ncsc1CNC(=O)CCc1nc2c(Br)cccc2[nH]1', 'Cc1ncsc1C[C@H](CS)C[NH+]1CCCCCC1', 'Cc1ncsc1C[NH+]1CC[C@H](Oc2ccccc2)C1', 'Cc1ncsc1C[NH2+][C@@H](C)c1ccccc1O', 'N#C/C(=C\\c1cccs1)C(=O)Nc1cc(Br)cc([N+](=O)[O-])c1', 'N#C/C(=C/c1cccs1)C(=O)Cn1cnc([N+](=O)[O-])c1', 'N#C/C(=C1\\N=C[C@H](S/C=N/S(=O)(=O)c2ccccc2)S1)c1ccccc1', 'Oc1ccc(/C=N/c2ccccc2Cl)cc1', 'Oc1ccc(/C=N/NC(=O)c2ccc(-c3ccccn3)s2)cc1', 'Oc1ccc(/C=N/Nc2nccc(N3CCCC3)n2)cc1Cl', 'Oc1ccc(/C=N/N2CCN(c3ccccc3)CC2)cc1', 'Oc1ccc(/C=N/NNc2ccc[n+]([O-])c2)cc1', 'Oc1ccc(/C=N/N[C@@H]2CCSC2)[nH]1', 'Oc1ccc([N+](=O)CCl)cc1OCC[NH+]1CCCCCC1', 'Oc1cc(-n2nc3ccccc3c2[N+](=O)[O-])cc(OCc2ccccc2)c1', 'NS(=O)(=O)N(C)CC(=O)N[C@H]1CCCC[C@@H]1OC1CCCCC1', 'NS(=O)(=O)N(C)[C@H]1CCC[C@H]1c1cccc(Cl)c1', 'NS(=O)(=O)N(C)c1ccccc1OCc1nc2ccc(Br)cc2[nH]1', 'NS(=O)(=O)N(C)[C@@H]1CCSc1Cl', 'Oc1ccc(F)c(-c2cc3[nH]c(=O)n(-c4ccccc4[N+](=O)[O-])c3s2)c1', 'Oc1ccc(F)c(Cl)c1Cn1ccnc1', 'Oc1ccc(F)c(F)c1SCCN1CC[NH+](Cc2ccccc2F)CC1', 'Oc1ccc(/C=C/Nc2ccc(S(=O)(=O)N3CCOCC3)cc2)cc1', 'C=CCn1cc(NC(=O)c2[nH]nc(C3CC3)c2C)c(C)n1', 'C=CCn1nc(SCC(=O)c2ccccc2)ccc1=O', 'C=CC#CCCNC(=O)Nc1ccc(-n2ncn(C)c2=O)cc1', 'C=CC[NH+](CC(=O)[O-])CC1CCCC1', 'C=CC[NH+]1CCC(CCC(=O)N2CCCCCC2)CC1', 'Clc1ccc(Br)cc1CCNC(=O)c1cccs1', 'Clc1ccc(Br)cc1', 'Clc1ccc(Br)cc1C(F)(F)C(F)F', 'Clc1ccc(Br)cc1OS(=O)(=O)c1cccnc1Cl', 'Clc1ccc(Br)cc1N1CCCCCC1', 'Clc1ccc(Br)cc1[C@@H]1COCC[NH+]1C[C@@H]1CCCO1', 'Clc1ccc(Br)cc1-c1cnccn1', 'Clc1ccc(Br)cc1S[C@H]1C[C@H](C2CC2)C1', 'Clc1ccc(Br)cc1Cl', 'Clc1ccc(Br)cc1/N=[NH+]/N=C1\\C(=O)C=C(c2cccnc2)S1', 'O=C(c1ccsc1)Nc1ccc(C(=O)Nc2ccccc2)cc1', 'O=C(c1ccsc1)N(Cc1ccccn1)c1nc2ccccc2c(=O)[nH]1', 'O=C(c1ccsc1)N(c1ccc2ccccc2c1)[C@@H]1CCO[C@@H]1c1ccc(Cl)cc1F', 'O=C(c1ccsc1)c1c(O)ccnc1SCC(F)F', 'O=C(c1ccsc1)c1ccc(Cl)c(C(F)(F)F)c1', 'O=C(c1ccsc1)c1cnc(-c2cccc(OC(F)F)c2)nn1', 'O=C(c1ccsc1)c1csc2ccccc12', 'O=C(c1ccsc1)c1c[nH]c2ccccc12', 'O=C(c1cc([S@](=O)CC(=O)Oc2ccccc2F)cs1)[NH+]1CCCC1', 'O=C(c1cn[nH]n1)N(CC(F)(F)F)[C@@H]1CC[C@H]([NH+]2CCCC2)C1', 'O=C(c1c[nH]cn1)OCc1ccc([N+](=O)[O-])c(Cl)c1', 'O=C(c1c[nH]cn1)Oc1cc(CNc2ccccc2)n(-c2ccccc2)n1', 'O=C(c1c[nH]cn1)O[C@@H]1CCS(=O)(=O)C1', 'O=C(c1c[nH]cn1)O[C@H]1CC(=O)N(C2CC2)C1', 'O=C(c1c[nH]cn1)O/N=C/C(Cl)=C/c1ccc([N+](=O)[O-])cc1', 'O=C(CN(C[C@H](O)c1ccccc1)C1CCCOCC1)N1C[C@@H]2C[NH+](CCCNC1=O)C2=O', 'O=C(CN(C[C@H]1CCCO1)C1CC1)Nc1cccc(F)c1', 'O[C@@H]1C(=O)N(Cc2cnc3ccccc3c2[O-])C=C[C@@H]1O', 'O[C@@H]1C(/C=N/O)C=CC(=O)C1(CO)CCc1ccccc1', 'O[C@@H]1C(c2ccc([N+](=O)[O-])cc2)=C(O)c2ccccc21', 'Clc1nncc(-n2cccc2)c1C1CC1', 'Clc1nncc(F)c1NC[C@H]1CCCO1', 'Clc1nncc(-c2ccccc2Cl)n1', 'Clc1nncc(-n2ccnc2)c1NC1CC1', 'Clc1nncc(-n2nc(N3CCc4ccccc43)C2)c1C1CC1', 'Clc1nncc(OCc2ccccc2F)n1', 'Clc1nncc(OC[C@@H]2Cc3cc(F)cc(F)c3O2)n1', 'Clc1nncc(OC2CCCC2)n1', 'Clc1nncn1CCNC(=O)Nc1nnc(-c2cccc3cccnc32)s1', 'Clc1nncn1Cc1cnn(-c2ccccc2)c1', 'Clc1nncn1C[C@@H](OCc1ccccc1)C1[C@@H](O)CO1', 'Clc1nncn1C1CCN(c2ccc(C(F)(F)F)nc2Cl)CC1', 'Clc1nncc(F)c1C(=O)N1CCCc2ccc(Br)cc21', 'Clc1nncc(S(=O)(=O)c2ccc(Br)cc2)n1', 'Clc1nncc(S/[N+]=C/C=CO)c1I', 'Clc1nncc(S/C(=C/C(=O)[O-])COC(F)(F)F)n1', 'Clc1cccc2nc(NC[C@@H]3CCCCO3)ncc1-2', 'Clc1cccc2ncnc(NCC[NH+]3CCC(O)CC3)c12', 'ClC(c1cc2ccccc2c2onc(-c3ccccn3)c12)[C@H]1NC(=O)N(Cc2ccccc2F)C1', 'ClC(c1cc2c(cc1Cl)OCCO2)c1ccc(F)cc1F', 'ClC(c1cc2cccc(Cl)c2o1)[C@H]1CCC[NH+]2CCC[C@@H]12', 'Clc1cccc(F)c1CCNC[C@]1(O)CCSC1', 'Clc1ccsc1F\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n', 'Clc1ccsc1[C@H](CSc1nnc(Cc2ccccc2)n1C(F)F)N1CCCC1', 'Clc1ccsc1[C@H](c1ccccc1)N1CCOCC1', 'Clc1ccsc1[C@H](O)CNC(=O)N[C@H]1CCCc2ccccc21', 'Clc1ccsc1[C@H](N[C@H]1CCOc2ccccc21)[C@H]1CCCC[C@@H]1C(=O)N1CCCC1', 'Clc1ccsc1[C@H](Cl)[C@@H]1CCOC2(CCCC2)C1', 'Clc1ccsc1[C@H]([O-])[C@@](C)(CO)c1ccccc1', 'Clc1ccsc1[C@H]([NH2+][C@@H]1CC[C@@H]([NH+](C)C)C1)c1ccccc1', 'N#C[C@@H]1CCCN(C(=O)[C@H]2C[C@@H]2c2ccc(F)c(F)c2)CC1', 'N#C[C@@H]1C(=O)NCCN1CC1CC[NH+](Cc2ccccc2)CC1', 'N#C[C@@H]1Cc2ccccc2N1C(=O)c1cccc(NC(=O)c2sc(C3CC3)nn2)c1', 'N#C[C@@H]1COCCCN1Cc1ccc(Br)s1', 'N#C[C@@H]1CNCCN1Cc1cc2c(cc1F)OCO2', 'N#C[C@@H]1C[NH+](Cc2ccc(F)cc2)C=C1c1ccc(Br)cc1O', 'Oc1cccc(-n2ccc(-c3cnc(-c4cc5ccccc5[nH]4)cc3)n2)c1', 'Oc1cccc(-n2c(CCNc3ncnc4ccccc34)nc3ccccc32)c1', 'Oc1cccc(Nc2snc3ccccc23)c1F', 'Oc1cccc(Nc2[nH+]c3ccc(F)cc3s2)c1', 'Oc1cccc(Nc2cc(Cl)ccc2Cl)c1', 'Oc1cccc(-n2c(SCC(=O)N3CCCC3)nc3ccccc32)c1', 'Oc1cccc(-n2c([O-])c3n(c2=O)CCCCCCC3)c1', 'Oc1cccc(F)n1\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n', 'Oc1cccc(SCn2nc3cccnc3n2)c1', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n', 'Oc1cccc(NCc2cccc(Cl)c2Cl)c1', 'Oc1cccc(S[C@@H](c2nccn2-c2ccccc2)c2ccccc2)c1', 'Oc1cccc(NC2=NC(=O)C3=Cc4ccccc4C(c4ccccc4)N32)c1', 'Oc1cccc(NC2CCC([NH2+]CCc3c[nH]nc3C)CC2)c1', 'Oc1cccc(NC2(c3ccccc3)CC2)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnc2c1cnn2-c1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnc2nc(C[S@@](=O)Cc3ccncc3)[nH]c12', 'Oc1cc(Cn2nc(I)c3ccccc3c2=O)ccc1N1CCC2(CC1)OCCO2', 'Oc1cc(Cn2nc(-c3ccc(Br)cc3)ccc2=O)ccc1Cl', 'Oc1cc(Cn2c[nH+]c(-c3ccccc3)c2NCc2ccco2)ccc1F', 'Oc1cc(Cn2cccn2)ccc1OCc1ccc(F)cc1', 'Oc1cc(COc2ncnc3c2cnn3Cc2ccccc2)on1', 'Oc1cc(Cn2ncc(Cl)c2[N+](=O)[O-])cnc1Cc1ccccc1', 'Oc1cc(Cn2nccc2-c2cccnc2)ccc1-c1cccnc1', 'Oc1cc(COc2n[nH]c(=O)[nH]2)ccc1C[NH2+]Cc1ccco1', 'Oc1cc(COc2ncnc(-c3cccs3)n2)ccc1O', 'Oc1cc(CCn2c(=O)[nH]c3ccccc32)c(Cl)cc1OCc1ccccc1', 'Oc1cc(CCNC2(CS)CCOCC2)ccn1', 'Oc1cc(CCNc2nc3ccccc3c(=O)n2-c2ccccc2)ccc1SCC(F)(F)F', 'Oc1cc(CCN[C@@H](c2ccc(F)cc2)c2ccc3c(c2)NC(=O)CCC3)ccn1', 'Oc1cc(O)ccc1S(=O)(=O)N1CCC[NH+](Cc2ccc(F)cc2)CC1', 'Oc1cc(Br)cs1\n\n\n\n', 'O=C(c1ccsc1COCC1CC1)N1CC[NH+](Cc2ccccc2)CC1', 'O=C(c1ccsc1CO)C(=O)NC[C@@H](c1cc2ccccc2[nH]c1=O)N1CCCC1', 'O=C(c1ccsc1COc1ccccc1)N1CCCc2ccccc21', 'O=C(/N=C/C=C\\c1cccc2c1OCO2)c1ccccc1', 'O=C(/N=C/CC1=Cc2ccccc2Oc2ccccc21)c1ccc(-c2cccs2)cc1', 'O=C(/N=C/C(=O)Nc1ccc(OC(F)(F)F)cc1)c1ccc([N+](=O)[O-])cc1', 'O=C(/N=C/CCC#N)c1ccccc1', 'O=C(/N=C/CC(=O)Cc1ccccc1)[C@@H]1CN(Cc2ccccc2)CCCO1', 'O=C(/N=C/CCc1ccccc1Br)Nc1ccc2c(c1)CCC2', 'O=C(/N=C/CC1CC[NH+](Cc2ccccc2)CC1)C1CCCC1', 'O=C(/N=C/CCOCc1ccccc1)NC1CC[NH+](Cc2ccccc2)CC1', 'O=C(/N=C/CCN1C(=O)c2ccccc2C1=O)NCc1cccc(Cl)c1', 'O=C(/N=C/CCSc1ccccc1)c1ccco1', 'Oc1cc(F)ccc1CN1CCN(S(=O)(=O)c2cccc(OCCn3cncn3)c2)CC1', 'Oc1cc(F)ccc1Oc1ccc(F)cc1', 'Oc1cc(F)ccc1NC[C@@H]1C[C@H]2CC[C@@H]1C2', 'Oc1cc(F)ccc1[C@@H]1CCCCN1C(=O)c1cc([N+](=O)[O-])cs1', 'Oc1cc(F)ccc1-c1cccc(S(=O)(=O)N2CCN(c3ccccc3)CC2)c1', 'Oc1cc(F)ccc1SCC(=O)N1CCCC1', 'Oc1cc(F)ccc1[C@@]1(COc2ccccc2F)CCO1', 'Oc1cc(Br)cc(CCc2nnc(-c3ccc(Cl)cc3)o2)c1', 'Oc1cc(Br)cc(OCCO)c1', 'Oc1cc(Br)cc(N2CCN(CC3CC3)CC2)c1', 'Oc1cc(Br)cc(Cl)c1C(F)(F)F', 'Oc1cc(Br)cc([C@H](c2ccccc2)N2CCOCC2)c1', 'Oc1cc(Br)cc(/C=C2\\CCC([C@H]3C=CC=N3)N(c3ccc(Cl)cc3)C2=O)c1O', 'Oc1cc(Br)cc([NH2+][C@H]2CCc3ccc([N+](=O)[O-])cc32)c1', 'Oc1cc(C[NH2+][C@H](c2ccccc2)c2ccc(F)cc2)c[nH]1', 'Oc1ccc(F)c(-c2csc(Cc3ccc(F)cc3)n2)c1', 'Oc1ccc(F)c(CCN2CC(=O)Nc3ncccc23)c1', 'Oc1ccc(F)c(C(=O)N2CCOCC2)c1', 'Oc1ccc(F)c(CNCCN2C(=O)CCCC2=O)c1', 'Oc1ccc(F)c(C[NH+]2CCC(Nc3ccc([N+](=O)[O-])cc3)CC2)c1', 'Oc1ccc(F)c(C/C(Cl)=C2\\CCCCN2)c1', 'Oc1ccc(F)c(C[NH2+][C@H]2CCCc3ccccc32)c1', 'Oc1ccc([N+](=O)[O-])c(-c2ccc(F)cc2)c1', 'Oc1ccc([N+](=O)[O-])c(-n2cccc2)c1', 'Oc1ccc([N+](=O)[O-])c(/N=N\\Nc2nnc(SCC(F)(F)F)s2)c1', 'Oc1ccc([N+](=O)[O-])c(/[NH+]=C/c2ccc(Cl)s2)c1', 'Oc1cc(F)cc(C[NH2+][C@@H]2CC[NH+](CC3CC3)C2)n1', 'Oc1cc(F)cc(NC2(c3cccc(F)c3)CCC2)c1', 'Oc1cc(F)cc(F)c1C(=O)Nc1ccc(C(=O)NC2CC2)cc1[N+](=O)[O-]', 'Oc1cc(F)cc([C@@H]2CSc3ccccc32)c1', 'Oc1cc(F)cc(/C=C/C[NH2+]Cc2ccc(F)cc2)c1', 'Oc1cc(F)cc([C@@]2(NC(=O)c3ccc[nH]c3=O)N3CCCC[C@H]32)c1', 'O=C(c1cccn2cnnc12)N1CCC[C@H](Oc2ccc(Cl)cc2)C1', 'O=C(c1ccccn1)N1CCO[C@H](c2ccoc2)C1', 'O=C(c1cccnc1)N(CC1CCN(C(=O)c2cnco2)CC1)C[C@H]1CCCO1', 'O=C(c1cccc2ncccc12)N1CCC(Cn2cc[nH+]c2-c2cccnc2)CC1', 'O=C(c1cn(-c2ccc(F)cc2Cl)nc1NC[C@H]1CN2CCCC[C@@H]2CO1)Nc1ccc(F)cc1', 'O=C(c1cncc([N+](=O)[O-])c1)N1[C@@H](CO)CC1(c1ccccc1)c1ccccc1', 'O=C(c1cncc(Cn2cncn2)n1)Nc1ccccc1Cl', 'O=C(c1cncc(N2C[C@@H](C(=O)NC3CC3)C2)n1)N1CCC[C@H](Cc2ccc[nH]2)C1', 'O=C(c1cncc(I)c1)N1CCC(OCc2ccccc2F)CC1', 'O=C(c1cncc(F)c1)N1CCc2sccc2[C@@H]1c1cccc(Br)c1', 'O=C(c1cncc(-c2ccccc2)n1)N1CC[C@@H](N2CCCC2=O)C1', 'O=C(c1cncc(Cl)c1)C1CCN(c2cc(-c3ccccc3)n[nH+]c2-c2cccs2)CC1', 'O=C(c1cncc(Br)c1)N1CCN(c2[nH+]ccn2Cc2ccccc2)CC1', 'O=C(c1ccsc1)Nc1ccn(CCOc2ccccc2F)n1', 'O=C(c1ccsc1)Nc1nnc(C(=O)Nc2ccc(-c3ccccc3)cc2)s1', 'O=C(c1ccsc1)N(CCNS(=O)(=O)c1ccccc1)N1CCOCC1', 'O=C(c1ccsc1)N(Cc1cccs1)[C@@H]1CCS(=O)(=O)C1', 'O=C(c1ccsc1)N(C1CC1)[C@@H](CCCn1cncn1)c1ccc(F)cc1', 'O=C(c1ccsc1)N(C[C@@H](O)c1ccccc1)C[C@@H]1CN2CCC[C@@H]2CO1', 'O=C(c1ccsc1)N(C[C@H]1COc2ccccc21)c1ccc(C(F)(F)F)cc1', 'O=C(COCCN=C(c1ccccc1)c1ccccc1)Nc1nccs1', 'O=C(COCCNC(=O)N(C1CC1)c1ccccc1)OC[C@@H]1CCCO1', 'O=C(COCCNc1ccccc1N1CCOCC1)NCc1ccccc1', 'O=C(COCCN1CCCC1=O)Nc1cc(Br)cc(Cl)c1', 'O=C(CNS(=O)(=O)c1cccs1)NC1CC1', 'O=C(COCCOC1CCCCCC1)Nc1ccccc1C(F)(F)F', 'O=C(COCCO)O[C@@H](c1ccccn1)c1ccccc1', 'O=C(COCCOc1ccc(Br)cc1)Nc1nc([O-])c(Cl)c2[nH]ccc12', 'O=C(CNS(=O)(=O)c1ccc(F)cc1)N[C@H]1CCSc2ccccc21', 'O=C(CNc1cc(F)c(F)cc1[N+](=O)[O-])Nc1ccccc1F', 'O=C(CNc1cccc(C(F)(F)F)c1)NC(=O)c1ccc(Cl)nc1', 'O=C(CNc1cc2c(cc1Cl)OCO2)c1ccccc1', 'O=C(CCSCc1[nH+]ccn1CC(=O)[O-])c1ccccc1', 'O=C(CCS(=O)(=O)c1cccc2ccccc12)OCc1ccccc1', 'O=C(CCS)NCc1ccc(C(F)(F)F)cn1', 'O=C(CNS(=O)(=O)c1ccccc1)Nc1cccc(-c2nc[nH]n2)c1', 'O=C(CNS(=O)(=O)/C=C\\c1ccc(F)cc1)Nc1ccc2c(c1)OCO2', 'O=C(CNS(=O)(=O)C1CCCCC1)NNc1cc(C#N)ccc1O', 'O=C(CNS(=O)(=O)c1cnc2c(c1)c(=O)n2Cc1ccccc1)NC1CCCCC1', 'O=C(CSCc1cccnc1)NN[C@@H]1CCc2ccccc21', 'O=C(CSCc1nnc2n1CCC2)NC1[C@@H](C#N)CCCc2ccccc21', 'O=C(c1ccc[nH]c1=O)N1N=C(c2cccs2)C[C@H]1c1cccnc1', 'O=C(/N=c1\\ocnn1Cc1ccccc1)c1cnn(C2CC2)c1', 'O=C(/N=c1\\[nH]c(S)nc2ccccc12)N1C[C@@H](O)[C@H]2C[C@@H]2[C@@H]2CCCC[C@H]21', 'O=C(c1c[nH]nc1-c1ccc(Br)cc1)N1CCN(S(=O)(=O)c2ccc(F)c(F)c2)CC1', 'O=C(c1c[nH]nc1-c1ccccc1F)N1CCO[C@@H](c2ccccc2)C1', 'O=C(c1c[nH]nc1-c1ccc2c(c1)COCO2)c1ccccc1Cl', 'O=C(c1c[nH]nc1-c1ccco1)N1CCC[NH+](Cc2ccccc2)c2ccccc21', 'O=C(c1c[nH]nc1-c1cccs1)N1CCC2(CC1)NCCN2C(=O)C[C@@H]1C(=O)N(Cc2ccccc2)CN1Cc1ccccc1', 'O=C(CNCc1noc(Cc2ccccc2)n1)N1CCN(c2ccccc2F)CC1', 'O=C(CNCc1nc2ccccc2c(=O)[nH]1)c1ccc(Br)cc1', 'O=C(CNCc1nnnn1-c1cc(F)cc(Cl)c1)NCc1ccc(F)cc1', 'O=C(CNc1nnc(C(F)(F)F)s1)Nc1ccsc1', 'O=C(CNc1nncc2c1nc(N)n2C)N1CCN(C2CCCCC2)CC1', 'O=C(Cc1nonc1C(F)(F)F)Nc1ccc(Cl)cc1', 'O=C(Cc1non1C1CC1)NCc1ccnc(-n2cccn2)c1', 'O=C(Cc1non2cccc12)N1CCN(CC(=O)/N=C\\c2ccccc2Cl)CC1', 'O=C(CN(c1cnn(C(F)F)c1)S(=O)(=O)c1ccccc1)N1CCc2ccc(Cl)cc2C1', 'O=C(C1CCN(C(=O)C2CCCC2)CC1)N1CCc2ccccc2C1', 'O=C(C1CCN(S(=O)(=O)c2ccc(F)cc2)CC1)N1CCN(C(=O)c2cccc(F)c2)CC1', 'O=C(C1CCN(c2ccc([N+](=O)[O-])c(F)c2)CC1)N1CCC[C@H]1CC(=O)Nc1ccccc1Br', 'O=C(C1CC1)N1CC(c2nc(C(F)(F)F)co2)C1', 'O=C(C1CC1)[NH2+]CCCc1ccccc1F', 'O=C(CNCc1csc2ccccc12)N1CC2(CCCCC2)Oc2ccccc21', 'O=C(CNCc1noc(COc2ccccc2)n1)N1CCOCC1', 'O=C(CNCc1nonc1C1CC1)N1CCC[C@H](c2nccn2C2CC2)C1', 'O=C(C1=c2ccccc2=[NH+][C@@H]1C[NH+](C)C(C)C)[C@@H]1CC(=O)N(c2ccccc2F)C1', 'O=C(CN(c1cn[nH]n1)S(=O)(=O)c1cccc(Br)c1)N1CCN(c2ccccc2)CC1', 'O=C(COc1cc(NC(=O)NC2(c3cccs3)CCOCC2)on1)N1CCCCC1', 'O=C(COc1cc(N(C(C)C)[C@H]2CCCC2=O)oc1C#N)N[C@@H](c1ccccc1)c1ccccc1', 'O=C(COc1cc(N)c(C(C)C)cc1[N+](=O)[O-])N1CCCC1', 'O=C(COc1cc(Nc2ccccc2)ccc1O)NNN1CCCCC1', 'O=C(COc1cc(N2CCCCC2)nn1-c1ccccc1F)N1CCN(S(=O)(=O)c2cccs2)CC1', 'O=C(CCCCCn1cncn1)N1CCC[C@@H](COc2ccc(F)cc2)C1', 'O=C(CCC(=O)c1cccc(Cl)c1)N1CCC[C@@H]1c1ccc([N+](=O)[O-])cc1', 'O=C(CCCc1ccc2[nH]c(=O)[nH]c(=O)c2c1)Oc1ccccc1', 'O=C(CCC1=CCCC1)N[C@H](Cc1ccccc1OC(F)F)c1ccccc1', 'O=C(CCCOc1ccccc1Br)N1CCCCCCC1', 'O=C(CCCNC(=O)C1(c2ccc(Br)cc2)CC1)Nc1ccccc1', 'O=C(CCCn1cnnn1)N1CCN(Cc2ccc(F)c(F)c2)CC1', 'O=C(CCCSC1CCCC1)NCc1nc2ccccc2[nH]1', 'O=C(CNc1nn(-c2ccccc2)c2ccccc12)Nc1cnn(Cc2ccccc2)c1', 'O=C(CNc1nn(Cc2ccccc2)c2ccccc12)N1CCN(c2ncccc2F)CC1', 'O=C(CNS(=O)(=O)/C=C/c1cccc(Br)c1)N1CCc2ccccc21', 'O=C(CNS(=O)(=O)/N=C/c1ccccc1)NCCc1ccc(Cl)cc1', 'O=C(CNS(=O)(=O)/c1c(Cl)cccc1Cl)NCCc1ccccc1', 'O=C(CNc1c(F)cc(F)cc1Cl)Nc1ccc(F)c(F)c1', 'O=C(CNc1c(-c2ccc(Cl)cc2)cnn1-c1ccccc1)NNC(=O)c1ccco1', 'O=C(CNc1c(Cl)cccc1[O-])Nc1ccc(C#CCCO)cc1', 'O=C(CNc1c(Br)cccc1[N+](=O)[O-])c1ccccc1', 'O=C(Cc1nnnn1Cc1ccccc1)Nc1ccncc1', 'O=C(CN(c1cnc(C(=O)NCc2ccccc2)c(Cl)c1)S(=O)(=O)c1ccoc1)c1ccccc1', 'O=C(CN(c1cncc(C(F)(F)F)n1)S(=O)(=O)c1ccccc1)N1CCCCCC1', 'O=C(CN(c1cncn(-c2ccccc2F)c1=O)NN1C(=O)COC1=O)c1ccc2c(c1)CCCC2', 'O=C(COc1cc(F)ccc1Br)N[C@@H]1CCSS1', 'O=C(CSCCNC(=O)NCc1ccc(F)cc1)Nc1ccc([N+](=O)[O-])cc1', 'O=C(CSCCNc1ccc2ccccc2n1)Nc1ccccc1OC(F)F', 'O=C(CSCCN1C(=O)COc2ccccc21)Nc1nccs1', 'O=C(COCc1cc(F)ccc1F)Oc1ccccc1', 'O=C(COCc1ccccc1)N1CCC(c2ccccc2)CC1', 'O=C(COCc1ccncc1)NCCN1C[C@@H]2CC[C@@H]1CN(S(=O)(=O)c1cccs1)CC2', 'O=C(CSCNc1ncccc1Cl)Nc1nc2ccc([N+](=O)[O-])cc2s1', 'O=C(C[NH+]1CC[C@@H]2CCCCC[C@@H]21)Nc1ccc(I)cc2cccc12', 'O=C(C[NH+]1CC[C@@H]2OCC[C@@H]2C1)Nc1ccc(F)cc1F', 'O=C(C[NH+]1CC[C@@H]2[C@@H](c3ccccc3)[C@H]2C1=O)NCc1ccccc1', 'O=C([C@H]1[C@H]2C=C[C@@H](CN2c2ccccc2)C1)N1CCC(Cn2ccnc2)CC1', 'O=C([C@H]1[C@H]2CCCC[C@@H]2C1=O)Nc1ccc(I)cc1', 'O=C([C@H]1[C@H]2C(=O)N(CCOCC(F)(F)F)C(=O)[C@H]12)c1cccs1', 'O=C([C@H]1[C@H]2C[C@@H](c3ccccc3O1)N(Cc1nc3ccccc3n12)C(=O)[O-])N1CCN(c2ccccc2)CC1', 'O=C(c1cc(F)c2[nH]ccc2n1)N1C[C@@H]2CC[C@@H](C1)N2C(=O)Nc1cccnc1Cl', 'O=C(c1cc(F)c2ccccc2c1Cl)Nc1ccc2c(c1)OCO2', 'O=C(c1cccs1)c1ccnc(OC2CCCCC2)c1', 'O=C(c1cccn2c(=O)cc(C3CC3)nc12)N1CCCC1', 'O=C(c1cccn2cc(-c3ccccc3)nc12)Nc1ccc(Cl)cc1', 'O=C(c1cccn2cnnc12)N(Cc1ccccc1)CC1CCOCC1', 'O=C(c1cccs1)CNc1ccc(Br)cc1', 'O=C(c1cccs1)C(=O)NCc1ncc(Cc2ccccc2)o1', 'O=C(c1cccs1)C1=CC(c2cccc(Cl)c2)SC(=S)N1c1ccccc1', 'O=C(c1cccc([N+](=O)[O-])c1)N1CCN(Cc2ccccc2)CC1', 'O=C(c1cccc(Cc2nnc(-c3ccccc3)o2)c1)N1CCOC(N2CCOCC2)C1', 'O=C(c1cccc(Oc2ncnc(-n3cccc3)c2)c1)N1CCCCCC1', 'O=C(c1cccc(I)c1)[C@H]1COc2ccccc2C1', 'O=C(c1cccc(NC(=O)N2CCN(c3ccc(F)cc3)CC2)c1)N1CCN(c2ccc(Cl)cc2)CC1', 'O=C(c1cccc(F)c1F)N1CCC(OCc2cccc(F)c2)CC1', 'O=C(c1cccc(-c2noc(-c3cc(F)ccc3F)n2)c1)[NH+]1CC[C@H]([NH+]2CCCC2)C1', 'O=C(c1cccc(S(=O)(=O)[N-]c2ccc(Cl)c(Cl)c2)c1)N1CCCCC1', 'O=C(c1cccc(Cl)c1)N1CCC[C@]2(CCN(S(=O)(=O)c3ccccc3)C2)C1', 'O=C(c1cccc(Br)c1)N1CCN(c2nccc([N+](=O)[O-])c2)CC1', 'O=C(c1cn[nH]c1)c1ccnc(N2CCC(OC(F)(F)F)CC2)c1', 'O=C(c1cn[nH]c1)OCC(=O)Nc1ccc(F)cn1', 'O=C(c1cn[nH]c1)N(C[C@@H]1CCCO1)c1ccccc1', 'O=C(c1cn[nH]c1)[C@@H]1C[C@@H]1c1ccc(Br)cc1', 'O=C(c1cn[nH]c1)[C@H]1CC[C@H](C(=O)[O-])C1', 'O=C(CS(=O)(=O)c1ccc(Cl)cc1)NCCc1ccccc1F', 'O=C(CS(=O)(=O)c1c[nH]c2ccccc12)NNC(=O)/C=C/c1ccco1', 'O=C(CS(=O)(=O)c1c(F)cccc1F)NC[C@@H](O)C1CCCCCC1', 'O=C(CS(=O)(=O)c1ccc(Br)cc1)Nc1cc(Br)on1', 'O=C(CS(=O)(=O)c1cnn(-c2ccccc2)c1)NCCc1c[nH]c2ccccc12', 'O=C(c1c[nH]c2cccc(F)c12)N1CCC[NH+](Cc2ccccc2)CC1', 'O=C(c1c[nH]c2ccccc12)N1CCC(OC[C@H]2CCCCO2)CC1', 'O=C(C1CCC1)[C@H]1CCCN(c2cc(C(F)(F)F)ccc2Cl)C1', 'O=C(CCOC1CCCCCC1)c1ccc(F)cc1', 'O=C(CCOc1cccc(Cl)c1)NCc1ccccc1Cl', 'O=C(CCOn1ccccc1=O)Nc1cc2c(cc1Cl)OCCO2', 'O=C(c1cn2ccsc2n1)C1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1', 'O=C(c1cn2ccsc2n1)N1CC(C2=Cc3ccc(F)cc3C2)C1', 'O=C(c1cn2ccsc2n1)c1ccc(S(=O)(=O)N2CCCCCC2)cc1', 'O=C(c1cn2ccsc2n1)[C@@H]1CCCCO1', 'O=[N+]([O-])c1cc(C(=O)N2CCCc3ccccc32)ccc1N1CCCC1', 'O=[N+]([O-])c1cc(SCCC(=O)N2CCC[C@@H](c3nc4ccccc4s3)C2)ncn1', 'O=[N+]([O-])c1cc(SC(CC2CC2)C(=O)[O-])ccc1Br', 'O=[N+]([O-])c1cc(SCc2ccco2)nc(Oc2ccccc2)c1', 'O=[N+]([O-])c1cc(SC2CCN(c3ccccc3)Cc3ccccc32)c2[nH]ccc2c1', 'O=[N+]([O-])c1cc(SC[C@@H]2Cc3ccccc3O2)c[nH]c1=O', 'O=C(CNCc1c(C)cc([N+](=O)[O-])c2ccc(Cl)cc12)c1ccccc1Cl', 'O=C(CNCc1c(O)cc([N+](=O)[O-])cc1F)Nc1cccc(O)c1', 'O=C(CNCc1c(F)cccc1F)N1CCN(Cc2ccc(F)cc2)CC1', 'O=C(CNCc1c(-c2ccccc2)noc1CO)NCc1ncoc1C1CC1', 'O=C(CNCc1c(Cl)cccc1Cl)OCC(=O)Oc1cccc(Cl)c1Cl', 'O=C(Cc1conc1C1CC1)NC[C@@H]1CC[C@@H]([NH2+]C(C)C)C1', 'O=C(Cc1conn1)N1CCC[C@@H](c2nc3ccccc3n2CC(F)(F)F)C1', 'O=C(C1CCN(c2ccccc2)CC1)c1ccc(-n2cccc2)cc1', 'O=C(C1CC[NH+](CCC(=O)N2CCNC[C@H]2CO)C1)N1CCN(c2ccccc2[N+](=O)[O-])CC1', 'O=C(C1CC[NH+]([C@H]2CC[NH+](Cc3c[nH]c4ccccc34)CC2)C1)NCCOc1ccccc1', 'O=C(C1CC[NH+]([C@@H]2CCN(C(=O)c3ccc4c(c3)OCCO4)CC2)[C@@H]1O)c1ccccc1', 'O=C(C1CCCCC1)N1CCC(N2CCOCC2)CC1', 'O=C(C1CCCCN1c1ccc2c(c1)COC2)NN1CCC([NH+]2CCCC2)CC1', 'O=C(C1CCCC1)N1CC[C@H]([N+]2=CCCC2)C1', 'O=C(/C(c1cccs1)=C1/N=C(c2ccc(Br)cc2)SC1=S)c1ccccc1', 'O=C(/C(c1cccs1)=N/NC(=O)[C@H]1C[C@@H]1c1cccc([N+](=O)[O-])c1)c1cccc(Cl)c1Cl', 'O=C(/C(c1cccs1)=[NH+]/C[C@H]1CC=Cc2sccc21)N1CCN(c2ccccc2)CC1', 'O=C(/N=C\\c1cc(Cl)ccc1N1CCOCC1)[C@H]1CC(=O)N(c2ccc(OC(F)(F)F)cc2)C1', 'O=C(/N=C\\c1ccco1)c1ccccc1[N-]Sc1ccccc1', 'O=C(CNS(=O)(=O)c1ccc([N+](=O)[O-])cc1)NCc1ccccc1', 'O=C(CNS(=O)(=O)CC1CCC1)Nc1ccc(N2CCOCC2)nc1', 'O=C(CNS(=O)(=O)C(F)(F)F)Nc1ccccc1Br', 'O=C(CNS(=O)(=O)Cc1ccccc1)Nc1ccc(F)c(F)c1F', 'O=C(CNS(=O)(=O)C1CCCC1)Nc1ccc(F)c(F)c1', 'O=C(CNS(=O)(=O)C[C@@H]1CCCO1)Nc1ccccc1F', 'O=C(CNc1nn2c(=O)cc(-c3ccc(Cl)cc3Cl)cc2s1)Nc1cccc2ncccc12', 'O=C(CNc1nn2cccc2c2ccccc12)Nc1cccc(-c2nncn2C2CC2)c1', 'O=C(CNc1nn2cnnc2c2c1CCCC2)NNC(=O)c1ccc(Br)cc1', 'O=C(CNc1nn(-c2cccc(C(F)(F)F)c2)c2c1NCCC2)Nc1ccc(F)c(F)c1', 'O=C(CNC(=O)N1CCC[C@@H](c2nc3ccccc3[nH]2)C1)NC1CC1', 'O=C(CNC(=O)COc1ccc(Cl)cc1F)NCc1cccs1', 'O=C(CNC(=O)Nc1nc2c(s1)CC[C@@H](C(=O)[O-])O2)c1ccccc1', 'O=C(CNC(=O)c1ccc([N+](=O)[O-])cc1)NNC(=O)c1ccccc1[N+](=O)[O-]', 'O=C(COc1cc(Cl)ccc1I)Nc1nc2c(s1)CO[C@@H](c1ccccc1)C2', 'O=C(COc1cc(C(F)(F)F)sc1-n1ccnc1)c1ccc(C#N)cc1', 'O=C(COc1cc(C[NH+]2CCOCC2)ccc1Br)NC[C@H](c1ccccc1)[NH+]1CCCC1', 'O=C(COc1cc(C2CC2)nc2ccccc12)N1CCCC[C@@H]1c1nc2ccccc2[nH]1', 'O=C(Cc1n[nH]c(=O)[nH]1)Nc1ccc2c(c1)oc(=O)n2Cc1ccccc1', 'O=C(Cc1n[nH]cc1C(=O)[O-])Nc1cc(C(F)(F)F)cc([N+](=O)[O-])n1', 'O=C(Cc1n[nH]c2ccccc12)Nc1ccc2c(c1)CCO2', 'O=C(CNc1cc2[nH]c(=O)[nH]c2cc1Br)N1CCC[C@@H]2CCC[C@H]21', 'O=C(CNc1cc2n(n1)CC[NH2+]C2)c1cccs1', 'O=C(/C(Cc1ccccc1)Cn1cccn1)c1ccc(Cl)cc1', 'O=C(/C(Cc1ccccc1)[NH+]1CCCC1)c1ccc([N+](=O)[O-])cc1', 'O=C(CC[C@@H](O)c1ccccc1)N[C@H]1CCCOc2ccccc21', 'O=C(CC[C@@H]1CN(C2CCCC2)CCN1Cc1ccccc1)Nc1ccccc1F', 'O=C([O-])/C=C/C(=O)N(Cc1ccccc1)c1ccccc1', 'O=C([O-])/C=C/c1cnn(-c2ccc(Br)cc2)c1N', 'O=C([O-])/C=C/Sc1cccc(F)c1[N+](=O)[O-]', 'O=C([O-])/C=C/[C@H]1Cc2ccccc2O1', 'O=C([O-])/C=C/[C@]1(CN2C(=O)Cc3ccccc32)CCC1=O', 'O=[N+]([O-])c1cc(Br)ccc1Oc1ccccc1C[NH2+]C[C@@H]1CCCO1', 'O=[N+]([O-])c1cc(Br)cc2cc(-c3ccc(Cl)c(Cl)c3)c(=O)[nH]c12', 'O=[N+]([O-])c1cc(C2CCCC2)nc2nnnn12', 'O=[N+]([O-])c1cc(C2(c3ccccc3Cl)CC2)cn1Cc1ccccc1', 'O=[N+]([O-])c1cc(C2=NN(Cc3ccccc3Cl)NC2=O)cc(Cl)c1Cl', 'O=[N+]([O-])c1cc(C2SCCS2)c2ccccc2c1[N+](=O)[O-]', 'O=[N+]([O-])c1cc(C2/N=CC=C[C@@H]2Oc2ccccc2)ccc1Br', 'O=[N+]([O-])c1ccc(O[C@H]2c3ccccc3C=C[C@@H]2O)cc1', 'O=[N+]([O-])c1ccc(O)c(NCc2cnn(C3CCCCC3)n2)n1', 'O=[N+]([O-])c1ccc(Oc2cc(C3CCCCC3)co2)cc1', 'O=[N+]([O-])c1ccc(S(=O)(=O)NCCCC(=O)N2CCCC[C@H]2Cn2cccn2)cc1', 'O=[N+]([O-])c1ccc(Sc2ccc(Cl)cc2)cc1', 'O=[N+]([O-])c1ccc(S[C@@H](C(=O)N2CCCC2)c2nccs2)cc1', 'O=[N+]([O-])c1ccc2[nH]nc(-c3ccccc3)c2c1', 'O=[N+]([O-])c1ccc2nccnc2c1C[NH2+]C[C@H]1CCCO1', 'O=[N+]([O-])c1ccc2ncn([C@@H](c3ncccc3Cl)C(F)(F)F)c2c1', 'O=[N+]([O-])c1ccncc1-c1ccc2c(c1)OCO2', 'O=[N+]([O-])c1ccncc1-n1cc([O-])c(NC2CC2)c1', 'O=[N+]([O-])c1ccncc1F', 'O=[N+]([O-])c1ccncc1OCCN1C(=O)c2ccccc2C1=O', 'O=[N+]([O-])c1ccncc1Oc1ccc(N2CCOCC2)nc1', 'O=[N+]([O-])c1ccncc1O[C@@H]1CCS(=O)(=O)C1', 'O=[N+]([O-])c1ccncc1O[C@H](Nc1ccc(Br)cc1)C(=O)NC1CCCCC1', 'O=[N+]([O-])c1ccncc1O/N=C1\\C(=O)Nc2ccccc21', 'O=[N+]([O-])c1ccncc1CC[NH+]1CCCCC1', 'O=[N+]([O-])c1ccncc1C(F)(F)F', 'O=[N+]([O-])c1ccncc1Cc1cccc(F)c1', 'O=[N+]([O-])c1ccncc1COC[C@H]1CCCC[C@@H]1n1cc[nH+]c1', 'O=[N+]([O-])c1ccncc1CN1C(=O)c2ccccc2C1=O', 'O=[N+]([O-])c1ccncc1Cn1cccn1', 'O=[N+]([O-])c1ccncc1CCl', 'O=[N+]([O-])c1ccncc1C[NH+]1CCN[C@H](c2ccc(Cl)c(Cl)c2)C1', 'O=[N+]([O-])c1ccncc1C[NH2+][C@@H]1CCN(C[C@@H]2CCCO2)C1', 'O=C(/N=C\\NCCc1cccc(Cl)c1)c1nc([O-])c2ccc(Cl)cn12', 'O=C(/N=C\\NC(=S)Nc1cccc2c1CCCC2)Nc1cc(Cl)ccc1Cl', 'O=C(/N=C\\NCc1nc2ccccc2s1)c1cccc2ccccc12', 'O=C(/N=C\\NC1=CN=NC1)\\CC(=O)[O-]', 'O=C(/N=C\\NCSc1cccc(Cl)c1)N1CCOC[C@@H]1C1CC1', 'Oc1cccc(-c2csc(COc3ccc(F)cc3)n2)c1', 'Oc1cccc(-c2csnn2)c1O', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(SC2CCC([NH2+]Cc3cccs3)CC2)c1', 'Oc1cccc(SC2(C(=O)c3ccccc3)CCOCC2)c1', 'Oc1cccc(SC2[C@H]3CCCC[C@H]23)n1', 'Oc1cccc(SC2[NH+]=c3ccccc3=[NH+]2)n1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(CN(Cc2ccccn2)c2ccc([N+](=O)[O-])cc2)c1NCC1CC1', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1N1CCOCC1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1-c1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1Cc1ccccc1Br', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1[C@@H]1CCS(=O)(=O)C1', 'O=[N+]([O-])c1c(F)cc(NCc2cccc(O)c2)cc1Cl', 'O=[N+]([O-])c1c(F)cc(N(C(=O)CN2CCOCC2)c2nc3ccccc3s2)c2ccccc12', 'O=[N+]([O-])c1c(F)cc(Nc2ncccn2)o1', 'O=[N+]([O-])c1c(F)cc(N2CCN(C(=O)COc3ccccc3)CC2)cc1O', 'O=[N+]([O-])c1c(F)cc(NN)c(O)c1O', 'O=[N+]([O-])c1c(F)cc(N[C@@H]2CCN(S(=O)(=O)OCCCn3cccc3)C2)cc1', 'O=[N+]([O-])c1c(F)cc(N/N=C/c2sc3ccccc3c2CO)cc1OCc1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccn1Cc1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccn1[C@H]1CCOc2ccc(F)cc21', 'O=[N+]([O-])c1c(-c2nsc([C@@H]3CCOC3)n2)cnn1Cc1ccccc1', 'O=[N+]([O-])c1c(Cl)n2nnc(C3CCCCC3)c2[nH]1', 'O=[N+]([O-])c1c(Cl)n2nnc(S[C@@H]3C=CC[C@@H]3CN3CCOCC3)c12', 'O=[N+]([O-])c1c(Cl)n2nnc(/N=C/c3ccccc3)c2nc1O', 'O=[N+]([O-])c1c([N+](=O)[O-])c(F)cc(Oc2cncc([N+](=O)[O-])c2)c1C1CC1', 'O=[N+]([O-])c1c(F)ccc(CNc2ncccc2F)c1Cl', 'O=[N+]([O-])c1c(F)ccc(Nc2ccccc2)c1', 'O=[N+]([O-])c1c(F)ccc(F)c1C(=O)NCCC[NH+]1CCCCC1', 'O=[N+]([O-])c1c([N+](=O)[O-])c(Br)cc(OCc2cccs2)c1C#CCOc1ccccc1', 'O=[N+]([O-])c1c(-c2cc([N+](=O)[O-])ccc2)cc[nH]1', 'O=[N+]([O-])c1c(-c2cc(=O)[nH]c3ccccc2c23CCOCC2)nccc1I', 'O=[N+]([O-])c1c(-c2cc(O)ccc2[N-]S(=O)(=O)c2ccc(Br)cc2)sc2c1CCCCC2', 'O=[N+]([O-])c1c(-c2cc(I)cc(Cl)c2)ncnc1N1CCc2ccccc21', 'O=[N+]([O-])c1c(-c2cc(F)ccc2O)n[nH]c1[O-]', 'O=[N+]([O-])c1c(-c2cc(-c3ccccc3)on2)sc2ncccc12', 'O=[N+]([O-])c1c(-c2cc(Cl)ccc2Cl)[nH]c2c1C[NH+](C[C@@H]1CCCCO1)CC2', 'Oc1ccc(/C=N/OC[NH+]2CCC[C@@H]2c2nc3ccccc3s2)cc1', 'Oc1ccc(/C=N/O)cc1Br', 'Oc1ccc(/C=N/Oc2cccc(Br)c2)cc1', 'Oc1ccc(/C=N/O[C@@H](c2ccc(F)cc2)C2CC2)cc1', 'Oc1ccc(F)c(Cl)c1COc1cnnc2ccccc12', 'Oc1ccc(F)c(-c2[nH+]c3ccc(Cl)cc3o2)c1', 'Oc1ccc(F)c(-c2cccc(F)c2F)c1', 'Oc1ccc(F)c(-c2noc(-c3ccsc3)n2)c1', 'Oc1ccc([N+](=O)Cc2ccccc2)cc1Cl', 'Oc1ccc([N+](=O)C=[N+]2NC(=O)N(c3ccccc3)c3ccccc32)cc1', 'Oc1ccc([N+](=O)C=C(Cl)Cl)cc1SCCO', 'Oc1ccc([N+](=O)C=O)cc1Br', 'Oc1ccc([N+](=O)C=NN2C(=O)N[C@](c3ccccc3)=N2)cc1', 'Oc1ccc([N+](=O)C=S)cc1', 'Oc1cc(-n2nnc(/N=C/c3ccccc3)c2-n2cncc2C(F)(F)F)ccc1F', 'Oc1cc(-n2nnc(C(F)(F)F)cc2=O)ccc1N1CCOCC1', 'Oc1cc(-n2nnc(Cc3ccc(Br)s3)n2)ccc1[O-]', 'Oc1cc(-n2nnc(C3CC3)n2)c(Cl)cc1Cl', 'Oc1cc(OC[C@@H](O)C[NH2+]C2CC2)ccc1OCc1cccnc1Cl', 'Oc1cc(OC[C@@H]2CCCO2)ccc1NCc1ccc(F)cc1', 'Oc1ccc([C@H](/C=C/c2ccc(Br)cc2)c2ccccc2)cc1', 'Oc1ccc([C@H](/N=C2\\Oc3ccccc3O2)c2c[nH]nc2-c2cc(F)ccc2F)cc1', 'Oc1ccc([C@H](NC(=O)c2cccc(Cl)c2)c2ccccc2)cc1', 'Oc1ccc([C@H](Nc2cc(Cl)ccc2F)N2CCCCC2)s1', 'Oc1ccc([C@H](NNC2CC[NH+](Cc3c[nH]c4ncccc34)C2)c2ccccc2)cc1', 'Oc1ccc([C@H](N[C@@H](Cn2cccn2)C2CC2)N2CCCC2=O)cc1', 'Oc1ccc([C@H](N[C@H]2CCC[C@@H]2CO)N2CCOCC2)cc1', 'Oc1cc(-n2cnnn2)ccc1N[C@@H](CO)Cc1ccccc1', 'Oc1cc(-n2cnnn2)c2cnn(CC3CCCCC3)c2n1', 'Oc1cc(C[NH+]2CCCC2)cc([N+](=O)[O-])c1O', 'O=C(CSCc1csc(NC(=O)c2ccc(Cl)cc2)n1)Nc1ccccc1', 'O=C(CSCc1c(N)cnn1Cc1ccccc1F)N/N=C/c1ccc2c(c1)OCO2', 'O=C(CSCc1cccc(Br)c1Cl)Nc1ccccc1C(=O)NCc1ccccc1', 'O=C(CSCC1=C2C=CC=C2C[C@H]1Cc1ccccc1)OCc1ccccc1', 'O=C(CSCC1(O)CCOCC1)c1cccc(F)c1', 'O=C(Cc1c([O-])c2cnccc2oc1=O)NNC(=O)Cn1ccc(=O)[nH]1', 'O=C(CSCNS(=O)(=O)c1cc(F)ccc1F)Nc1ccccc1F', 'O=C(CSCNSc1ccccc1)c1ccccc1F', 'O=C(COCCC(=O)[O-])NCC1(c2ccc(F)cc2)CC1', 'O=C(COCCC12CC3CC(CC(C3)C1)[C@H]2C([O-])Cl)N1CCCCCC1', 'O=C(COCCCOc1ccccc1)NCCC12CC3CC(CC(C3)C1)C2', 'O=C(COCCCNC(=O)C1(c2ccccc2)OC1=O)c1ccccc1', 'O=C(COCCC[C@@H]1COc2ccccc2O1)NCc1ccc2c(c1)CCCC2', 'O=C(COCCCCl)N1CCC[C@H]1c1ccccc1', 'O=C(COCCC[NH+]1CCCC1)Nc1cc2c(cc1Cl)OCCO2', 'O=C(COCCC[NH3+])Nc1ccccc1F', 'Oc1cccc2cc([C@@H]3Nc4ncccc4C(=O)O3)c(O)cc1O[C@H]2c1cccc(Cl)c1', 'O=C([C@H](Nc1ncccn1)C1CC1)NC1(CC(F)(F)F)CCCC1', 'O=C(c1c[nH]c(=O)[nH]c1=O)N1CCN(c2ccccc2O)CC1', 'O=C(c1c[nH]c(=S)s1)N1CCC(OCc2cccc(F)c2)CC1', 'O=C(c1c[nH]nc1N(c1ccccc1)c1cccc(F)c1)c1cccc(F)c1', 'O=C([C@H](c1c(F)cccc1Cl)N1CCOCC1)N1CCCCC1', 'O=C([C@H](c1ccc(F)cc1)N1CCOCC1)N(CCO)Cc1c[nH]c2c(Cl)cccc12', 'O=C([C@H](c1ccc(Cl)cc1)N1CC[NH2+]CC1)NCCOC1CC1', 'O=C([C@H](c1ccc([O-])c(C(F)(F)F)c1)N1CCCCCC1)N1CCCS1(=O)=O', 'O=C([C@H](c1ccc(Br)cc1)N1CCN(Cc2c[nH]c[nH+]2)CC1)c1ccccc1F', 'O=C(c1cc(Br)c2c(c1)OCCO2)N1CCC[C@@H]1c1cccc(F)c1', 'O=C(c1cc(Cl)cc(OC(F)F)c1)N1CCOCC1', 'O=C(c1cc(Cl)ccn1)N1CCOCC1', 'O=C(c1cc(Cl)ccc1Cl)N(Cc1ccccc1)Cc1ccccn1', 'O=C(c1cc(Cl)cc2c1OCCO2)c1ccccc1', 'O=C(c1cc([N+](=O)[O-])ccc1F)N1CCC([NH2+]CCN2CCCCC2)CC1', 'O=C(c1cc(Br)cs1)N1CCC(OCc2ccccc2)CC1', 'O=C(c1cc(Br)cnc1Cl)N1CCC(c2ccccc2)=N1', 'O=C(c1cc(Br)cn1CC(F)(F)F)N1CCC[C@H](C(N2CCCCC2)c2ccccc2F)C1', 'O=C(c1cc(Br)cn2cnnc12)N1CC[NH+]2CCCC[C@H]2C1', 'O=C(c1cnn2cccnc12)N1CCC[C@H]1c1cccs1', 'O=C(c1cnn2c1NCCCC2)N1CC(c2ncon2)C1', 'O=C(CSCCOCc1ccccc1)Nc1n[nH]c2ccccc12', 'O=C(CSCCO)Nc1ccccc1NCc1cccc(Cl)c1', 'O=C(CSCCOc1ccccc1-c1ccccc1)NNC(=O)c1cnc2ccccc2n1', 'O=C(CNS(=S)(=O)c1ccc(Br)cc1)Nc1cccc([N+](=O)[O-])c1', 'O=C(CSCc1csc2ccccc12)Nc1ccc2[nH]c(=O)n(-c3ccccc3)c2c1', 'O=C(CSCc1csnn1)NCc1ccc(-c2nc3ccccc3o2)cc1', 'O=C(CN(Cc1ccccc1)c1ccccc1)Nc1nncs1', 'O=C(CNS(=S)(=O)c1ccccc1)Nc1ccc(F)c(Cl)c1', 'O=C(CSCN1C(=O)CCc2ccccc21)NCCCc1ccccc1', 'O=C(CSCNOCc1ccccc1)N1CCCC1', 'O=C(CSCNOc1ccccc1Cl)Nc1ccc(O)cc1', 'O=C(CSCNO[C@@H]1CCCOc2ccccc21)N[C@H]1CCc2ccccc21', 'O=C(Cc1c[nH]nc1-c1ccccn1)N[C@H](CO)c1ccccc1', 'O=C(CN(CCC(N)=O)C1CCCCC1)Nc1ccccc1', 'O=C(CN(CC(F)F)c1ccccc1)Nc1ccc2c(c1)C(=O)NCC2', 'O=C(CN(CCc1cccnc1)C1CC1)Nc1ccccc1C(F)(F)F', 'O=C(CN(CC1CC1)C1CC1)Nc1cccc2cccnc12', 'O=C(CN(CCO)c1ccccc1)Nc1ccc(Cl)cc1', 'O=C(CN(CCn1cccn1)c1ccccc1)NC[C@@H]1CCOC1', 'O=C(CN(CCS)Cc1ccccn1)c1cccc([N+](=O)[O-])c1', 'O=C(CN(CC[NH+]1CCCC1)C1CCCCC1)N1CCCC1', 'O=C(Cc1noc(C2CCCCC2)n1)N[C@@H]1CCSc2ccccc21', 'O=C(Cc1noc2ccccc12)Nc1cccc(Sc2nc3ccccc3s2)c1', 'O=C(C1=CCCCN1)N1CCN(c2ccc([N+](=O)[O-])c(NN)n2)CC1', 'O=C(C1=CC(=N/O)CC1)c1ccccc1', 'O=C(C1=CCN(c2ccc(C#N)c(Cl)c2)CC1)N(Cc1ccccc1)Cc1ccccc1', 'O=C(C1=CC[C@@H](c2ccccc2)/C(=C\\c2cccs2)C1)N1CCC(O)[C@H](O)C1', 'O=C(C1=CC[NH+](CCCN2CCOCC2)CC1)Nc1cccnc1', 'O=C(/C=C1/cc[nH]1)NCC1(N(C)C(=O)c2ccccc2)CCCCC1', 'O=C(/C=C1/c2ccccc2S1)[C@]1(Nc2ccccc2)SC2=c3ccccc3=[NH+][C@@H]21', 'O=C(c1ccsc1)c1n[nH]c2ccccc12', 'O=C(c1ccsc1)c1nn(-c2ccc(F)cc2)c2ccccc12', 'O=C(c1ccsc1)c1nn(Cc2ccc(F)cc2)c2c1CCCC2', 'O=C(c1ccsc1)c1nnc(-c2ccccc2)s1', 'O=C(c1ccsc1)c1nnn2c1Nc1ccc(Cl)cc1C(=O)N2CCc1ccccc1', 'O=C(Cc1c[nH]nc1-c1ccccc1)[C@H]1CCC[NH+](Cc2c[nH]c3ccccc23)C1', 'O=C(CC[NH+]1CCC(O)(c2ccccc2)CC1)Nc1cccc2ncccc12', 'O=C(CNS(=S)(=O)c1ccccc1)NC1CC1', 'O=C(CNCc1nc(-c2ccccn2)no1)N1CCN(C[C@@H]2CCCO2)CC1', 'O=C(CNCc1nccs1)N1CCC(Oc2ccc(F)cc2F)CC1', 'O=C(CNCc1nc2ccccc2[nH]1)Nc1cc(C(=O)[O-])cc(Cl)c1', 'O=C(CNCc1ncoc1Cc1ccncc1)Nc1ccc(Cl)cn1', 'O=C(CNCc1nc[nH]n1)N1CC[NH+]([C@H]2CCOC2)CC1', 'O=C(c1c[nH]c([N+](=O)[O-])c1)N1CCO[C@@H](c2ccccc2F)C1', 'O=C(c1c[nH]c2ccccc12)N1CCN(c2ccc[nH+]c2)CC1', 'O=C(c1c[nH]c([N+](=O)[O-])c1)Oc1cc2c(cc1Cl)OCCC[C@@H]2O', 'O[C@@H](c1csnc1)c1cc(F)c(F)cc1Cl', 'O[C@@H](c1csnn1)[C@H]1C[NH+]2CC[NH+]1CCC2', 'Oc1cc(-n2cn/cc2)ccc1N[C@@H]1CCC[C@H](C2CC2)C1', 'Oc1cc(-n2cn/nc3c4c(cccc24)OC[C@@H]3CO)cc1=O', 'O=[N+]([O-])c1cc(OCCC(=O)NC2CC[NH+](Cc3ccccc3)CC2)ccc1Cl', 'O=[N+]([O-])c1cc(OCc2ccccc2)c(Cl)cc1OCO', 'O=[N+]([O-])c1cc(OC2CCCC2)c2ccccc2c1F', 'O=[N+]([O-])c1cc(OCOCC[C@H]2CCCCO2)ccc1Cl', 'O=[N+]([O-])c1cc(OC[C@@H]2CCCO2)cs1', 'O=[N+]([O-])c1cc(OC[C@H]2CCOC2)nc(-c2nccn2C)c1', 'O=[N+]([O-])c1ccc(NC[C@@H]2C[C@@H](O)C[NH+]2Cc2ccccc2)cc1F', 'O=[N+]([O-])c1ccc(N2CCN(Cc3ccc3c(c3)CCCC3)CC2)nc1', 'O=C([O-])c1[nH]ncc1CNC(=O)C[C@@H]1CCCC[NH2+]1', 'O=[N+]([O-])c1cc(Cl)cc(N2CCC([NH2+]CC[C@@H]3COCCO3)CC2)c1', 'O=[N+]([O-])c1cc(Cl)ccc1Cl', 'O=C(CCc1ccccc1)N1CCC(NC(=O)Nc2ccnc(F)c2)CC1', 'O=C(CCc1nnc2ccccn12)Nc1c2[nH]c(=O)c3ccccc3c2c(Cl)cc1Cl', 'O=C(CCc1n[nH]c(=O)c2ccccc12)NNC(=O)c1ccc(F)cc1F', 'O=C(CCc1nc(-c2ccccc2)cs1)Nc1ccc(F)cc1Cl', 'O=C(CCc1nnc(-c2ccccc2)o1)NCCC[NH+]1CCCC1', 'O=C(CC[NH+]1CCOCC1)N[N+](=O)[O-]', 'O=C(/C(c1cccs1)CN1CCOCC1)C(=O)Nc1cccc(Br)c1', 'O=C(/C(c1cccs1)CC1(C)CC1)Nc1ccccc1Br', 'O=C(/C(c1cccs1)C(=O)Nc1ccc(F)c(F)c1)c1cccs1', 'O=C(/C(Cc1ccccc1)c1cccc(F)c1)c1ccccc1', 'O=C(/C(Cc1ccco1)c1ccc(F)c(Cl)c1)Cn1c[nH+]ccc1=O', 'O=C(/C(Cc1cccnc1)Nc1nc(-c2cccs2)cs1)[C@@H]1COc2ccccc2N1', 'O=C(/C(Cc1ccc[nH]1)c1cccc(F)c1)c1ccc(Cl)cc1', 'O=C(Cc1nn[nH]n1)Nc1ccc(C(=O)N2C[C@@H](C)O[C@H](CCc3ccccc3)C2)cc1', 'O=C(Cc1noc2ccccc12)Nc1ccccc1', 'O=C(Cc1noc2nc(-c3ccccc3)oc12)Nc1ccc(F)cc1', 'O=C(Cc1nc(=O)n(-c2ccc(Cl)cc2)[nH]1)NCc1ccc(F)cc1', 'O=C(Cc1nc(N2CCOCC2)nc2c(C(F)F)ncn12)Nc1ccc([N+](=O)[O-])cc1', 'O=C(Cc1nc(Cl)ccc1Cl)c1cccc(Cl)c1', 'O=C(Cc1nc([O-])c(-c2ccccc2Cl)n1C1CC1)Nc1ccccc1', 'O=C(CS(=O)(=O)[N-]C1CCCC1)Nc1nc2ccccc2s1', 'O=C(CS(=O)(=O)[N-]c1cc(F)ccc1SCc1cccs1)Nc1ccccc1', 'O=C(Cc1coc2ccccc12)NCc1cccc(-n2cccn2)c1', 'O=C(Cc1coc2nccc(NC(=O)Cc3ccccc3)c12)c1ccc(Cl)cc1', 'O=C(Cc1cn(-c2ccccc2)nn1)NCc1cccc(CN2C(=O)[C@H]3CCCC[C@H]3C2=O)c1', 'O=C(Cc1cn(CC2CCC2)c(=O)[nH]1)Nc1cn[nH]c1', 'O=C(Cc1nn(-c2ccccc2)c2c1CCC2)c1ccccc1', 'O=C(Cc1nn(Cc2ccccc2)cc1COc1ccc(Br)cc1)c1ccccc1', 'O=C(Cc1n[nH]c2c1CC[NH2+]CC2)N1CCc2ccccc21', 'O=C(Cc1n[nH]c2ncccc12)N1CCC[C@H](c2nc(C(F)(F)F)cs2)C1', 'O=C(CNS(=S)(=O)c1ccccc1Cl)NCC(F)(F)F', 'O=C(c1cc([N+](=O)[O-])[nH]n1)N(Cc1ccc(Cl)cn1)[C@H]1CCS(=O)(=O)C1', 'O=C(CS(=O)(=O)[C@H](Cc1ccccc1)N1C(=O)c2ccccc2C1=O)c1ccco1', 'O=C(CS(=O)(=O)[C@H]1CCCc2ccccc21)N/N=C/c1cnc2ccccc2c1', 'O=C(CS(=O)(=O)[C@H]2CCOc3ccccc32)N1CCc2ccccc21', 'O=C(/C=C/c1ccc(COc2ccc(Cl)cc2)o1)Nc1cc(Cl)ccc1Cl', 'O=C(/C=C/c1cccc(Cl)c1)NCc1cc(Br)ccc1Br', 'O=C(/C=C/c1ccc2c(c1)OCO2)c1cnn(Cc2ccccc2)c1', 'O=C(/C=C/c1ccco1)N1CCC[C@H](C2(O)COC2)C1', 'O=C(/C=C/c1cccnc1)Nc1ccc([N+](=O)[O-])c(Br)c1', 'O=C(/C=C/c1cccs1)N1CCN(c2ccc(Cl)cc2Cl)CC1', 'O=C(/C=C/c1c(-c2ccccc2)[n+](-c2ccccc2)c([O-])[nH]c1=O)Nc1cccc(Cl)c1', 'O=C(/C=C/c1c(F)cccc1Cl)Oc1ccccc1[N+](=O)[O-]', 'O=C(/C=C/OCCOc1ccc(Oc2ccccc2)cc1)Nc1ccc(Cl)cc1', 'O=C(/C=C/OCc1ccncc1)c1c(Cl)cccc1Cl', 'O=C(/C=C/OC1CCCC1)Nc1ccccc1[N+](=O)[O-]', 'O=C(/C=C1CS(=O)(=O)C[C@H]1O)c1ccccc1Br', 'O=C(/C=C/c1c(NC2CC2)oc2ccccc12)NNC1CCCC1', 'O=C(/C=C/c1c(NN)cc(C#N)c(=O)c2ccccc12)Nc1ccc(Oc2ccccn2)cc1', 'O=C(/C=C/c1c(N[C@@H]2CCSc3ccccc32)[nH]c2ccccc12)NCc1ccc(C(=O)OCCO)o1', 'O=C(/C([O-])(c1ccc(F)cc1)C(F)(F)F)c1c(Cl)cccc1Cl', 'O=C(/C([O-])(F)F)c1ccc(OC(F)(F)F)cc1OCO[C@@H]1CCc2cc(OCc3ccccc3)ccc21', 'O=C(/C=C1/NCCN1C(=O)Cc1ccc(Cl)cc1)Nc1cn[nH]c1', 'O=C(/C=C1/N(Cc2ccccc2)c2ccccc2C1=O)Nc1ccccc1OC1CCCC1', 'O=C(/C=C1/N=NC(=O)N(C2CCC2)C1=O)c1ccc2c(c1)COC2', 'O=C(/C=C1/NNc2ccccc2N1)N1CCC(OC[C@H]2CCCCO2)CC1', 'O=C(/C=C1/N[C@H](c2ccccc2)SC1=S)c1ccc(F)cc1', 'O=C(/N=c1\\o/c(=C\\c2ccc(F)cc2)[nH]c1=O)c1c(Cl)cccc1Cl', 'O=C(/N=c1\\occcc1OCCCCc1ccccc1Cl)c1cnn(-c2ccccc2)c1', 'O=C(/N=c1\\onc2n1CCC2)N1CCN(c2ccccc2)CC1', 'O=C(/N=c1\\occc(-c2ccccc2)n1)c1cccc(F)c1', 'O=C(/N=c1\\oc2ccccc2[nH]1)c1cccc(C(F)(F)F)c1F', 'O=C(/N=c1\\[nH]c2c(ccc3ccccc13)[C@@H]1CCCN12)c1cccc(F)c1', 'O=C(c1ccc[nH]c1=O)N(CC1=CCCCC1)C1CCCCC1', 'O=C(c1cccnc1Cl)N1CCC(CCNc2cccc(Cl)c2)CC1', 'O=C(c1cccnc1SCC(=O)Nc1cccc(Cl)c1)N[C@@H]1CCCC[C@@H]1CO', 'O=C(c1cccnc1N1CC[NH+](Cc2ccc(Cl)cc2)CC1)N1CCCCC1', 'O=C(c1cccnc1)N1CCc2cc(Cl)ccc21', 'O=C(c1cc(Cl)cnc1N1CCCCC1)N1CCN(C(=O)c2cccnc2)CC1', 'O=C(c1cc(Cl)cn1-c1nccc(C(F)(F)F)c1-c1ccccc1)c1cccc(C(F)(F)F)c1', 'O=C(c1cc(Cl)cn2cnnc12)N1CCN(Cc2ccccc2)CC1', 'O=C(c1cc(F)cc(F)c1)N1CCC(n2cncn2)CC1', 'O=C(c1cc(F)ccc1F)N1CCC[C@@H]1[C@@H]1CCCC[C@@H]1O', 'O=C(c1cc(F)cc2cc[nH]c12)N1CCO[C@@H](c2ccccc2)C1', 'O=C(c1ccc[nH]c1=O)Oc1ccc(Br)c([N+](=O)[O-])c1', 'O=C(c1ccc[nH]c1=O)/C(=C/c1ccc(Cl)c([N+](=O)[O-])c1)OCC(F)(F)F', 'O=C(c1ccc[nH]c1=O)N(Cc1nc2ccccc2n1Cc1ccccc1)S(=O)(=O)c1ccccc1', 'O=C(c1ccc[nH]c1=O)[C@@H]1CCC[NH+](Cc2ccc(Cl)cc2)C1', 'O=C(c1cnc(Nc2ccccc2)s1)N1CCC[C@H](c2cc[nH+]cc2)C1', 'O=C(c1cnc(N2CCc3ccccc32)nc1)N1CC[C@@H]([NH+]2CCCC2)C1', 'O=C(c1cnc(NSc2ncccn2)nc1Cl)[C@H]1CCCO1', 'O=C(c1cnc(N[C@H]2CCOC2)cn1)Nc1ccc(Cl)c2ccccc12', 'O=C(c1cnc(N/N=C/c2ccc(Cl)cc2)o1)N1CCc2ccccc21', 'O=C(CNc1nnccn1)Nc1c(F)cccc1F', 'O=C(CNc1nnccc1[N+](=O)[O-])Nc1ccc(CCC(=O)N2CCCCCC2)cc1', 'O=C(c1c[nH]c2nc[nH]c(=O)c12)N(Cc1ccccc1)c1ccccn1', 'O=C(c1c[nH]c2nc[nH]c12)N1CCN(c2ccc(F)cc2)CC1', 'O=C(c1c[nH]c2cc([N+](=O)[O-])ccc12)[C@@H](Cc1ccccc1)n1cccn1', 'O=C(CCS(=O)(=O)Cc1ccc(Cl)cc1)NC1CCCC1', 'O=C(CCS(N)=C(Cl)Cc1ccccc1)NC[C@@H](O)c1ccc(F)cc1', 'O=C(CCc1c[nH]c2ccccc12)N[C@H]1CCC[C@H](O)C1', 'O=C(CCc1ccc[nH]1)Nc1c(F)cccc1F', 'O=C(CC[C@@H]1COc2ccccc2C1)NCCc1ccc(Cl)cc1', 'O=C(CC[C@@H]1NC(=O)c2ccccc2N1CC1[NH+]=c2ccccc2=[NH+]C1=O)N[C@H]1CCCC[C@@H]1O', 'O=C(CC[C@@H]1c2ccccc2C(=O)N1Cc1cccc(C(F)(F)F)c1)N1CCN(c2cccc[nH+]2)CC1', 'O=C(CC[C@@H]1[NH+]=c2ccccc2=[NH+]C1)N(Cc1cccs1)SC(F)(F)F', 'O=C(/C(c1cccs1)c1ccccc1)[C@@H](c1ccccc1O)c1ccccc1', 'O=C(/C(c1cccs1)SC1=NC=N1)c1ccc(Cl)cc1', 'O=C(/C(c1cccs1)S(=O)(=O)c1ccc(Cl)cc1)c1ccc(F)cc1', 'O=C(/C(c1cccs1)Sc1ccccc1)c1ccno1', 'O=[N+]([O-])c1cc(Br)c2[nH]cnc2n1', 'O=[N+]([O-])c1cc(Br)c2nccc(=O)n2c1N', 'O=[N+]([O-])c1ccc(/C=C\\c2cc(Br)ccc2O)cc1O', 'O=[N+]([O-])c1ccc(/N=C\\c2nc(Nc3cccc(F)c3)c(=O)[nH]c2[O-])cc1', 'O=[N+]([O-])c1ccc2oc(C(=O)N3CCN(c4ccccc4[N+](=O)[O-])CC3)nc2c1', 'O=[N+]([O-])c1ccc2onc(Nc3ncccc3Cl)c2n1', 'O=[N+]([O-])c1ccc([C@H](CCCO)N[C@H]2CC(c3ccccc3)=NO2)c2ccccc12', 'O=[N+]([O-])c1ccc([C@H]2CSC[C@@H](Cc3cccs3)O2)c(Cl)c1', 'O=C(C1CCN(C(=O)c2ccccn2)CC1)N1CCCC1', 'O=C(C1CCN(Cc2cccc(Cl)c2)CC1)N1CCC2(CC1)OCCO2', 'O=C(C1CCN(C2CCCC2)CC1)N1CCC2(CCSCC2)CC1', 'O=C(CNS(=O)(=O)c1ccccc1C(F)(F)F)N[C@H](c1ccccc1)c1ccc([N+](=O)[O-])cc1', 'O=C(c1ccsc1)[C@@H]1CC(=O)N(Cc2cccc(F)c2)C1', 'O=C(c1ccsc1)[C@@H]1Cc2ccccc2C(=O)O1', 'O=C(c1ccsc1)[C@@H]1COc2ccccc2O1', 'O=C(c1ccsc1)[C@@H]1CN(C[C@@H]2CCCO2)CCN1C[C@H]1CCCO1', 'O=C(c1ccsc1)[C@@H]1C[C@@H](c2ccccc2)[NH2+]Cc2ccccc21', 'O=C(c1ccsc1)[C@@H]1CS[C@@H]2[C@H](C=C3C=CC=CC3=N2)C1', 'O=[N+]([O-])c1cc(N2CCC[C@@H]2c2nc3ccccc3s2)cc([N+](=O)[O-])c1', 'O=[N+]([O-])c1cc(N[C@H](c2cccc(F)c2)C2CCCC2)c[nH+]c1', 'O=[N+]([O-])c1cc(F)c(Br)cc1Cl', 'O=[N+]([O-])c1cc(F)cc(CNc2ccccc2F)c1', 'O=[N+]([O-])c1cc(F)c2cccnc2c1Oc1ccccc1', 'O=[N+]([O-])c1cc(Cl)n(-c2ccccc2)c1NC[C@@H]1CCCO1', 'O=[N+]([O-])c1cc(F)cc(CN2CCOC[C@H]2Cn2cnnn2)c1', 'O=[N+]([O-])c1cc(F)ccc1C[NH+]1CCC(c2ccccc2)=N1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1OC1CC1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1', 'O=[N+]([O-])c1ccc2oc(SCc3ccc(Br)o3)nc2c1', 'O=[N+]([O-])c1ccc2occc2c1-c1ccccc1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1-c1ccc(F)cc1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1O', 'O=[N+]([O-])c1ccc2cc[nH]c2c1CN1CCN(c2ccccc2)[C@@H]1c1ccc(-c2ccc(F)cc2)cn1', 'O=[N+]([O-])c1ccc2cc(=O)oc(=O)c(-c3cccc([N+](=O)[O-])c3)c2c1', 'O=[N+]([O-])c1ccc2cccnc2c1C[NH+]1CCC2=C(N2c2ccccc2)CC1', 'O=[N+]([O-])c1cc(Br)c(C(=O)N2CCN(c3ccccc3[N+](=O)[O-])CC2)cc1OC(F)F', 'O=[N+]([O-])c1cc(Br)c(O)c([N+](=O)[O-])c1OCCC[NH+]1CCCC1', 'O=[N+]([O-])c1cc(Br)c(NC2CC2)cn1', 'O=[N+]([O-])c1cc(Br)c(F)c(/C=N/Nc2cn(C)c3ccccc23)c1', 'O=[N+]([O-])c1cc(Br)c(-c2cccc(Br)c2)cc1O', 'O=[N+]([O-])c1cc(Br)c(S(=O)(=O)Nc2ccc(F)cc2)c2c1CCO2', 'O=[N+]([O-])c1cc(Br)c(Cl)cc1[C@@H](O)c1ccccc1', 'O=[N+]([O-])c1cc(Br)c([O-])cc1NCC(F)(F)F', 'O=[N+]([O-])c1cc(Br)c(Br)c(C(F)(F)F)c1O', 'Oc1ccc(F)cc1OCCNC(=O)c1ccccc1Cn1cccn1', 'Oc1ccc(F)cc1Oc1ccc(NCc2ccc(Cl)nc2)cc1', 'Oc1ccc(F)cc1O[C@@H](CO)C(F)(F)F', 'Oc1ccc(F)cc1OS(=O)(=O)c1ccccc1O', 'Oc1ccc(F)cc1O[C@H](C(=O)NC[C@H]1CCCO1)c1ccccc1', 'Oc1ccc(F)cc1[C@H](CO)NC1CC[NH+](C2CC2)CC1', 'Oc1ccc(F)cc1[C@H]1CCCN(c2nccs2)C1=O', 'Oc1ccc(/N=c2/[nH]c(C(F)(F)F)ccc2-c2cccc(Br)c2)cc1[N+](=O)[O-]', 'Oc1ccc(/N=c2/[nH]ccn2CCOc2ccccc2)cc1', 'Oc1ccc(/N=c2/[nH]c(-c3cccc(Cl)c3)n[nH]2)c(Br)c1', 'Oc1ccc(/N=c2/[nH]c3ccccc3[nH]2)cc1N1CCCC1', 'O=C(CSCc1cc(=O)[nH]c(-c2ccccc2)n1)NC[C@H]1CCCO1', 'O=C(CSCc1ccc(Cl)c(Cl)c1)NNC(=O)c1ccc(S(=O)(=O)N2CCCC2)cc1', 'O=C(CNS(=O)(=O)c1ccc2ccccc2n1)N1CCN(S(=O)(=O)c2ccccc2)CC1', 'O=C(CNS(=O)(=O)/c1ccc2ccccc2c1)N1CCN(c2ccc(F)cc2)CC1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1C(=O)N[C@H]1CCSc2ccc(F)cc21', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1OC[C@@H]1CCCO1', 'O=[N+]([O-])c1c(Br)cnc(Oc2ccccc2Cl)c1C(F)c1ccccc1', 'O=[N+]([O-])c1c(Br)cncc1Sc1ccc(C[NH3+])s1', 'O=[N+]([O-])c1c(Br)cnc2nccn12', 'Oc1ccccc1Oc1ccccc1-c1nc(-c2ccccc2)nn1CC1CC1', 'Oc1ccccc1COC1CCN(C(=O)Nc2ccccc2Oc2ccc(F)cc2)CC1', 'Oc1ccccc1COc1ccccc1C(=O)NC[C@H]1CCCc2ccccc21', 'Oc1ccccc1CO[C@@H]1CCCC[C@@H]1[C@@H]1CCCN1C(=O)c1ccccc1', 'Oc1ccccc1Oc1c(Cl)ccc2c1NCCN2C(=O)c1ccccc1', 'Oc1ccccc1Oc1cccc[n+]1/N=C/c1nc2ccccc2o1', 'Oc1ccccc1Oc1cncc([O-])n1', 'Oc1ccccc1Cc1ccccc1Br', 'Oc1ccccc1C1CCN(C(=O)CN2c3ccccc3CC2)[C@H]2CS(=O)(=O)C[C@@H]21', 'Oc1ccccc1C1(O)CCCCC1', 'Oc1ccccc1C12CC(CC(COc3ccccc3)CS2)[C@@H]1S', 'Oc1ccccc1C1=C(N/N=C/c2ccccc2)C=C1c1cc[nH]c1', 'Oc1ccccc1C1OCOc2cc(S(=O)(=O)NCc3ccccc3)ccc21', 'Oc1ccccc1C1NC[C@@H](c2cccs2)Cc2cccnc21', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(-c2cc(F)c(Br)c(Cl)c2)c1', 'Oc1cccc(-c2ccc(C(=O)Nc3cccc(F)c3)nc2)c1', 'Oc1cccc(-c2cc3occc3nc2NC2CCC(CO)CC2)c1', 'Oc1cccc(-c2ccnc(NCCOc3ccccc3)c2)c1', 'Oc1cccc(-c2cc[nH]c2)c1CCC(=O)Nc1ccccc1Cl', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(-n2c(Sc3ccccc3)n[nH]c2=O)c1', 'Oc1cccc(-n2c(S[C@H]3CCOC3)nc3ccccc32)c1', 'Oc1cccc(Oc2ccccc2C(F)(F)F)c1', 'Oc1cccc(O[C@@H](C(=O)[O-])c2ccc(Oc3ccccc3)cc2)c1', 'Oc1cccc(O[C@@H]2CCOc3ccc(Oc4ccccc4F)cc32)c1', 'Oc1cccc(N2CCN(C(=O)c3ccc(C[NH+]4CCC(C(=O)Nc5cccc(Cl)c5)cc4)cc3Cl)CC2)c1', 'Oc1cccc(N2C(=O)[C@@H](Cc3ccccc3)N2c2ccc(C(=O)[O-])cc2)c1', 'Oc1cccc(N2C[C@@H](C(=O)N3CCc4sccc4[C@@H]3c3ccccc3)CC2=O)c1', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(CCl)c1F', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(C(=O)Nc2nc(-c3ccc(F)cc3F)cs2)c1', 'Oc1cccc(C(F)(Cl)NNC(=O)c2csc(Br)c2)c1', 'Oc1ccccc1O[C@@H](CC(F)(F)F)c1cc(F)cc(F)c1', 'Oc1ccccc1O[C@@H](N[C@@H](Cc1ccccc1)c1ccccn1)NC1(c2ccccc2)CC1', 'Oc1ccccc1O[C@@H](c1ccccc1)c1cccc(F)c1', 'Oc1ccccc1O[C@@H]([C@@H]1CCCN(c2ncccn2)C(F)(F)F)C1', 'Oc1cccc2ncn(/C(CO)c3nc(-c4ccccc4F)n[nH]3)c12', 'Oc1cccc2ncn(/N=C/c3ccccc3)c(=O)c12', 'Oc1cccc2ncn(/c3c(C(=O)NCCc4ccccc4)c3ccccc2)c1', 'Oc1cccc(O[C@H](C(=O)[O-])C[NH+]2CCN(Cc3ccccc3)CC2)c1', 'Oc1cccc(O[C@H]2CCCCO2)c1', 'Oc1ccc(Oc2cn(Cc3ccccc3)nn2)cc1Cl', 'Oc1ccc(Oc2cnccc2C2CC2)cc1F', 'Oc1ccc(Oc2cn(-c3ccccc3)ccc2=O)nc1O', 'Oc1ccc(Oc2cn(Cl)nc2-c2c[nH]c3ccccc23)cc1', 'O=C([O-])c1nn(C(=O)c2ccc(Br)cc2)c2ncc(C(F)(F)F)cc12', 'O=C([O-])c1nnc([C@H](c2cccnc2)C2CC2)s1', 'O=C([O-])c1nn2c(-c3cccnc3)cc(-c3ccc(Cl)cc3)nc2c1-c1ccccc1F', 'O=C([O-])c1nnnn1-c1cccc(OCC(F)F)n1', 'O=C([O-])c1nnsc1Sc1ccccn1', 'O=C([C@H](CNC(=O)c1ccc(N2CCOCC2)cc1)c1ccc(F)cc1)NNC(=O)c1ccoc1', 'O=C([C@H](CNc1ncccn1)[NH+]1CCCC1)NN[C@@H]1CCS(=O)(=O)C1', 'O=C([C@H](CN1CCOCC1)N1CCN(c2cccc(Cl)c2)CC1)C[NH+](CC1CC1)C1CC1', 'O=C([C@H](Sc1noc2ccccc12)C1CCCC1)NC[C@H](O)c1ccccc1', 'O=C([C@H](Sc1n[nH]c2ccccc12)c1cc2c(cc1F)OCCO2)N1CCSCC1', 'O=C([C@H](Sc1ncccn1)C(=O)NC1CC1)NCc1cc(F)ccc1F', 'O=C(Cc1c(NC2CC2)c[nH]c(=O)c1Cl)NC[C@H]1CCc2ccccc21', 'O=C(Cc1c(N[O-])c2ccccc2c(=O)n1C1CCCC1)N1CCN(Cc2ccc3c(c2)OCO3)CC1', 'O=C(Cc1c(-c2ccccc2)nn([O-])c1Cl)C(=O)N1CCN(c2ccccc2)CC1', 'O=C(Cc1coc(-c2cccs2)n1)N1CCN(C[C@H]2CCCO2)CC1', 'O=C(Cc1coc([C@@H]2CCCO2)c1)Nc1cccc2ccccc12', 'O=C(CSCC(=O)N1CCCCC1)NNC(=O)c1ccccc1C(F)(F)F', 'O=C(CSCC(N)=N/O)[C@H]1CCC(=O)O1', 'O=C(CSCC(F)(F)F)N[C@@H](C(=O)c1cccc(Cl)c1)N1CCCC1', 'O=[N+]([O-])c1c(F)cccc1CN1CCCCC[C@H]1c1cccc(F)c1', 'O=[N+]([O-])c1c(F)cccc1OCCN1CCN(C(=O)c2cccs2)CC1', 'O=[N+]([O-])c1c(F)cccc1NC1CC[NH+](Cc2ccc(Cl)cc2)CC1', 'O=[N+]([O-])c1c(F)cccc1[C@@H](O)c1cc(F)ccc1F', 'O=[N+]([O-])c1c(F)cccc1Cl', 'Oc1ccc(F)c(F)c1CNc1cccc2c1CCCC2', 'Oc1ccc(F)c(Cl)c1', 'Oc1ccc(F)c(Cl)c1CCc1ccc2ccccc2n1', 'Oc1ccc(F)c(Cl)c1OC1CC[NH2+]CC1', 'Oc1ccc(F)c(Cl)c1N[C@@H]1CCS/C1=N/c1cccc(Br)c1', 'Oc1ccc(F)c(Cl)c1Sc1ccc(Br)cc1O', 'Oc1ccc(F)c(Cl)c1Cl', 'Oc1ccc([N+](=O)c2ccco2)cc1O', 'Oc1ccc([N+](=O)c2ccc(Br)o2)cc1', 'Oc1ccc([N+](=O)c2ccccc2)c(Oc2cccc(F)c2)c1', 'Oc1ccc([C@H]([O-])Cc2ccccc2)c(F)c1', 'Oc1ccc([C@H](OC(=O)Cc2ccccc2)N2CCOCC2)c2c1CCCC2', 'Oc1ccc([C@H](O)c2ccco2)c(F)c1', 'O=C([C@H](C(=O)NC1CC1)N[C@@H]1CC(C)(C)c2ccccc21)N1CCN(c2ccc([N+](=O)[O-])cc2F)CC1', 'O=C([C@H](C(Cl)F)N1CCN(CC(=O)N2CCOCC2)CC1)Nc1ccccc1', 'O=C([C@H](C(F)(F)F)[NH+]1CCN(CCO)CC1)Nc1ccccc1C(F)(F)F', 'O=C([O-])CSCC[C@@H]1CCCO1', 'O=C([O-])CS(=O)(=O)Cc1nccn1Cc1c(F)cccc1Cl', 'O=C([O-])CSc1ccc2c(c1)CCC(=O)N2', 'O=C([O-])CS[C@H]1[NH+]=c2ccccc2=C1C1CC1', 'O=C([O-])CS/N=C1\\C(=O)Nc2c1cnc1ccccc21', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(Nc2c(C(F)(F)F)ncnc2/N=C/c2ccccc2Cl)c1', 'Oc1cccc(Nc2ccc(F)c3c2[nH]c2ccccc3c2=O)c1[O-]', 'O=[N+]([O-])c1cc(Cl)n(-c2ccccc2F)n1', 'O=[N+]([O-])c1cc(Nc2ccccc2Cl)c2ccc(F)cc2n1', 'O=[N+]([O-])c1cc(Nc2ncccc2F)ccn1', 'O=[N+]([O-])c1cc(Cl)nc(Nc2ccccc2Cl)c1', 'O=[N+]([O-])c1cc(Cl)ncc1NC[C@H](SCc1ccccc1)c1ccccc1', 'O=[N+]([O-])c1cc(Cl)nc2cnccc12', 'O=[N+]([O-])c1cc(C(=O)NC[C@H]2CCC[NH2+]C2)[nH]c1N1CC=C(c2ccc(F)cc2)CC1', 'O=[N+]([O-])c1cc(O[C@@H]2CCCC[C@H]2[C@@H]2CCC[NH2+]C2)c(Cl)cc1Cl', 'O=[N+]([O-])c1ccc(CCNC2=Cc3ccccc3CS2)cc1', 'O=[N+]([O-])c1ccc(C(=O)N2N=C/C(=C/c3ccccc3Cl)C2=O)cc1', 'O=[N+]([O-])c1ccc(Cc2nc(-c3ccco3)no2)c(F)c1', 'O=[N+]([O-])c1ccc(COc2ccc(CC[NH+]3CCOCC3)cc2)cc1', 'O=[N+]([O-])c1ccc(CN2CCc3[nH]c[nH+]c3/C(=N/OCc3cccs3)C2)cc1', 'O=[N+]([O-])c1ccc(CSc2nc(-c3ccc(Cl)cc3Cl)ncn2)cc1', 'O=[N+]([O-])c1ccc(C[NH+]2CCC(c3nc4ccccc4o3)CC2)cc1', 'O=[N+]([O-])c1ccc(C[NH2+]Cc2ccc(O[C@@H]3CCOCC3)o2)cc1', 'Oc1cc(Br)cs1\n\n\n\n\n', 'Oc1cc(Br)cs1\n\n\n\n\n\n', 'Oc1ccc([N+](=O)[O-])c(Sc2cc(F)ccc2F)c1', 'Oc1ccc([N+](=O)[O-])c(S[C@@H](C(=O)[O-])c2ccccc2Cl)c1', 'Oc1ccc(F)cc1-c1csc([C@@H]2Oc3ccccc3O2)[nH+]1', 'O=C([C@H]1CCCC[C@@H]1C[NH3+])[NH+]1CCCC1', 'O=C([C@H]1CC(=O)N(C2CCCC2)C1)N1CCCC[C@H]1CO', 'O=C([C@H]1CCOC2(CCOCC2)C1)N1CCN(C(=O)[C@@H]2C[C@@H]2c2cccc(F)c2)CC1', 'O=C([C@H]1CC[C@@H]2CCCC[C@H]2[NH2+]1)N1CCOCC1', 'O=C([C@H]1CC[C@H](C[NH3+])O1)c1ccc(Br)cc1', 'Oc1cc(C=O)ccc1OCCN1CCOC[C@@H]1C1CC1', 'Oc1cc(Cn2cccc2)ccn1', 'Oc1cc(Cn2ccnn2)c2cn[nH]c2c1O', 'Oc1cc(Br)cs1\n\n\n\n\n\n\n', 'Oc1cc(C=O)on1', 'Oc1cc(C=O)nc(-c2ccccc2)n1', 'Oc1ccc([N+](=O)[O-])c(NCc2ccccc2)c1', 'Oc1ccc([N+](=O)[O-])c(Nc2cccc(Cl)c2)c1', 'Oc1ccc([N+](=O)[O-])c(N2CCOCC2)c1F', 'Oc1ccc([N+](=O)[O-])c(N[C@@H]2CC(=O)c3ccc(Cl)cc32)c1', 'Oc1ccc([N+](=O)[O-])c(N[C@H]2CCc3cc(F)ccc32)c1', 'Oc1ccc([N+](=O)[O-])c(N/N=C\\c2ccccc2F)c1', 'Oc1ccc([N+](=O)c2cccc(C(F)(F)F)c2)cc1', 'Oc1ccc([N+](=O)c2ccccc2Cl)c(I)c1', 'Oc1ccc([N+](=O)c2ccc([N+](=O)[O-])cc2)cc1Oc1cccc2c1OCCO2', 'Oc1ccc([N+](=O)c2ccc(C(=O)[O-])cc2F)cc1', 'Oc1ccc([N+](=O)c2ccc(O)cc2)cc1OCc1ccccc1', 'Oc1ccc([N+](=O)c2ccc(F)cc2F)cc1', 'Oc1ccc([N+](=O)c2ccc(Cl)cc2Cl)cc1', 'Oc1ccc([N+](=O)c2ccc(Br)cc2)cc1Cl', 'Oc1ccc(/C=C(/SCc2cccc(C#N)c2)NN[C@H]2CCCO2)c([N+](=O)[O-])c1', 'Oc1ccc(/C=C(/Cl)c2ccccc2[N+](=O)[O-])cc1', 'Oc1ccc(/C=C(/C#N)c2cnc(Cl)cn2)cc1', 'Oc1ccc(/C=C(/Nc2cccc(F)c2)c2ccccc2)cc1', 'Oc1ccc(Oc2cc([N+](=O)[O-])ccc2N2CCOCC2)cc1', 'Oc1ccc(Oc2cc(F)cc(F)c2)cc1', 'Oc1ccc(Oc2cc([C@@H]3OCCN(c4ccccc4OC(F)(F)F)n3Cc3ccccc3)ccc2F)cc1', 'Oc1ccc(Oc2cc(-c3ccccc3)ncn2)cc1O', 'Oc1ccc(Oc2cc(Cl)ccc2N2CCOCC2)cc1', 'Oc1ccc(Oc2cc(Br)ccc2F)cc1', 'Oc1ccc(Oc2cnc(Oc3ccccc3)nc2-c2ccccc2)cc1', 'Oc1ccc(Oc2cnc3c(c2)C[C@H](c2ccc(Br)s3)cc2)C1', 'Oc1ccc(Oc2cncc(-c3ccccc3COc3ccccc3)c2)cc1', 'Oc1ccc(O[C@@H]2C[C@@H](C(=O)[O-])[C@H]2C(=O)Nc2ccccc2)cc1', 'Oc1ccc(O[C@@H]2C[C@@H]3CCCC[C@H]3C2)cc1OCC(F)(F)F', 'Oc1ccc(O[C@@H]2C[C@@H]2C(=O)O[C@H]2CCCc3ccccc32)cc1[N+](=O)[O-]', 'Oc1ccc([N+](=O)[O-])c(I)c1O', 'Oc1ccc(F)c(S(=O)(=O)[O-])c1', 'Oc1ccc([N+](=O)[O-])c(Cl)c1Oc1ncc(Cl)c(Cl)c1Cl', 'Oc1ccc([N+](=O)[O-])c(Cl)n1', 'Oc1ccc(F)c(/N=C2\\S[C@H](c3ccco3)N=N2)c1', 'Oc1ccc(/C=N\\CC[NH+]2CCCCC2)cc1', 'Oc1ccc(/C=N\\C(=O)COCc2ccc(F)cc2Cl)c(F)c1', 'Oc1ccc(/C=N\\Cc2ccccc2)c(Br)c1O', 'Oc1ccc(/C=N\\C2=C(C(=O)c3nncn3CCc3ccncc3)[NH+](C)C2)cc1', 'Oc1ccc(/C=N\\C[C@@H]2CSc3ccccc32)cc1', 'Oc1ccc([N+](=O)C2=C([O-])O[C@@H]2c2ccccc2)cc1OCC(F)(F)F', 'Oc1ccc([N+](=O)C2CC2)cc1', 'Oc1ccc([N+](=O)C2(CCO)Oc3ccccc32)c2ccccc12', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(Br)c1OCC1(O)CCCCC1', 'Oc1cccc(Br)c1OCc1c[nH]c2nccc(Oc3ccccc3C(F)(F)F)c12', 'Oc1cccc(Br)c1OC1(c2ccccc2)CCCC1', 'Oc1cccc(Br)c1OC[C@@H]1CN2CCC[C@@H]2CO1', 'Oc1cccc(Br)c1OC[C@H](O)CBr', 'Oc1cccc(Br)c1OCBr', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(CCCC(=O)c2ccccc2F)c1', 'Oc1cccc(CC(=O)N2CCC[C@H]2[C@@H]2CCCC2=O)c1', 'Oc1cccc(CCc2cncc(-c3ccccc3)n2)c1', 'Oc1cccc(CC2(O)C[C@@H]2c2ccco2)c1O', 'Oc1cccc(CCNc2c([N+](=O)[O-])c(O[C@@H](C)c3ccccc3)no2)c1', 'Oc1cccc(CC[C@@H]2CCCCO2)c1', 'Oc1cccc(CCn2ccnc2NC[C@@H](O)C[NH+]2CCCC2)c1', 'Oc1cccc(CC[C@H]2CCCC[NH2+]2)c1', 'Oc1cccc(CC[NH2+]Cc2cccc(C(F)(F)F)c2)c1', 'Oc1ccccc1[C@@H]1CCCN1S(=O)(=O)/C=C/c1ccco1', 'Oc1ccccc1[C@@H]1C(c2cccs2)=CN=C1c1ccc([N+](=O)[O-])cc1', 'Oc1ccccc1[C@@H]1Cc2c([nH]c3ccccc32)c2ccccc21', 'Oc1ccccc1[C@@H]1COc2ccccc2N1CN1CCOCC1', 'Oc1ccccc1[C@@H]1CN(c2ccccc2[N+](=O)[O-])CCN1CCO', 'Oc1ccccc1[C@@H]1C[C@@H](C[NH2+][C@H]2CCc3ccccc32)c2ccccc2S1', 'Oc1ccccc1[C@@H]1C[C@H](O)C[NH+]1Cc1ccc([N+](=O)[O-])cc1C(F)(F)F', 'Oc1ccc(F)cc1-n1ccc2cc(F)ccc21', 'Oc1ccc(F)cc1-n1cccc1CO/C=C/c1ccccc1Cl', 'Oc1ccc(F)cc1[C@H]1CCCN1c1ccccc1', 'Oc1ccc(F)cc1[C@H]1Oc2ncccc2Oc2ccccc21', 'Oc1ccc(F)cc1[C@H]1[C@@H]2C(=O)N(c3cccnc3)[C@@H]2[C@@H]2C=CC[C@H]21', 'Oc1ccc(F)cc1[C@H]1[C@H]([NH2+][C@H]2CCOc3ccc(Cl)cc32)OC1(CCc2ccccc2)c1ccc(F)cc1', 'Oc1cccc2cc(/C=N\\C(=O)c3ccc4c(c3)OCCO4)oc12', 'Oc1cccc2cc(/C=C/C(=O)N[C@@H]3OCCO3)ccc12', 'Oc1cccc2cc(/C=N/c3ncnc4[nH]ncc34)ccc12', 'Oc1cccc2cc(C(=O)Nc3ccc(F)cc3)cn-2/c([O-])c1', 'OC[C@@H]([NH2+]Cc1ccccc1Br)C(F)(F)F', 'OC[C@@H]([NH2+][C@H]1CCOC2(CCCCC2)C1)c1ccccc1', 'OC[C@@H]([NH2+][C@@H]1CCOC2(CCCC2)C1)C(=O)NCc1ccccc1', 'Oc1ccc([N+](=O)c2cc(Br)ccc2C(F)(F)C[NH+]2CCCC2)cc1', 'O=[N+]([O-])c1c([O-])[nH]c(-c2cccc3ccccc23)c1Nc1cccc(Cl)c1', 'O=[N+]([O-])c1c([O-])[nH]c(=O)[nH]1', 'O=[N+]([O-])c1c([O-])[nH]c(C[NH+]2CCC(n3cccn3)CC2)c(-c2ccccc2)c1=O', 'O=[N+]([O-])c1cc(N2CCC(c3cc[nH]c3)CC2)ccc1OCC(F)(F)F', 'O=[N+]([O-])c1cc([C@H]2SCC(=O)N(Cc3cccs3)C2=O)ccc1OS(=O)c1ccccc1Cl', 'O=[N+]([O-])c1cc([C@H]2S[C@@H](O)C[C@H]3CC[C@H]2O3)cc(Br)c1O', 'O=C(/N=c1c(N2CCN(c3ccccc3)CC2)nc2cc3c(cc12)CCCC3)c1ccco1', 'O=C(/N=c1c(N)nc2c(Cl)cccc12)c1ccc[nH+]c1', 'O=C(C1CC1)N(CCO)Cc1ccccc1Cl', 'O=C(C1CC1)N1CCCN(Cc2cccc(Cl)c2)CC1', 'Oc1cc(F)ncc1-c1ncc(Cl)cc1O[C@H](CO)C(=O)[O-]', 'O[C@@H]1[C@@H](CC(=O)NNc2cnc3ccccc3n2)CCN1c1ccccc1', 'O[C@@H]1[C@@H](C(=O)[O-])[C@@H]1CC[C@@H]1CCCO1', 'O[C@@H]1[C@@H](Cc2ccccc2)N/C1=C[C@@H]2c1ccccc1C2=O', 'O[C@@H]1[C@@H](C2CC([O-])O[C@@H]2c2ccc(Cl)cc2)CC1', 'O[C@@H]1[C@@H](CO)CN(c2ccc(Cl)cc2)C1', 'O[C@@H]1[C@@H](CNC2CCCCC2)CCCN(Cc2ccco2)C1', 'O[C@@H]1[C@@H](C[C@@H]2CCCO2)C(=O)C[C@@H]1c1ccc2c(c1)OCO2', 'O[C@@H]1[C@@H](C[NH+]2CCC[C@H](CSc3cc4ccccc4[nH]3)C2)CCCN1', 'COc1nnn(-c2ccc(OC)cc2)c1C(C)C', 'COc1nnn(-c2ccc(NC(=O)[C@H](C#N)c3cccnc3)cc2)c1', 'COc1nnnc(NC(=O)Cc2ccc(I)cc2)n1', 'COc1nnnc(N(Cc2ccccc2)CC(=O)c2ccc(Cl)cc2)c1O', 'COc1nnnc(Nc2c(Cl)cccc2Cl)c1C', 'COc1nnnc(N2CC[C@@H](Cc3csc(C(=O)NCCCOC)n3)C2)c1C', 'COc1nnnc(N[C@H](C)c2ccc(F)c(F)c2)c1Cl', 'COc1nn[nH]c1C(C)(C)OC(=O)Cn1c(=O)n(-c2ccc(C)c(C)c2)c(C)cc1=O', 'COc1nn[nH]c1CCCNC(=O)c1ccnc(Br)c1', 'COc1nn[nH]c1CC(=O)/C=C/c1cccc([N+](=O)[O-])c1', 'COc1nn[nH]c1CCc1nnc2n1CC[NH+](Cc1cccc(C(F)(F)F)c1)CC2', 'COc1nn[nH]c1CC1CC[NH+]([C@@H](C(=O)N2CCCCC2)c2ccccc2)CC1', 'CCn1nc(C)c([C@@H](NC(=O)Nc2ccc(Cl)c(C#N)c2)c2ccccc2)c1C', 'CCn1ncc2cccc(CC(=O)NCc3ccccn3)c2c1=O', 'CCn1nc2n(c1=O)CCC[C@@H]2[NH2+][C@H]1c2ccccc2C[C@H]1OC', 'CCn1ncnc1C[C@H](NC(=O)c1cnn(-c2ccc(C)cc2)c1)C(C)C', 'CN(C)C(=O)C(=O)Nc1ccc(-c2nc3ccccc3s2)c(C(F)(F)F)c1', 'CN(C)/C=C/C(=O)Nc1ccccc1OCc1cccc(Cl)c1', 'CN(C)c1ncccc1C(=O)Nc1ccc(N2CC[NH+](CC(Cl)(Cl)Cl)CC2)cc1', 'CNS(=O)(=O)c1ccc(C(=O)NC[C@@H](c2ccc(Cl)c(Cl)c2)[NH+](C)C)cc1', 'CN(Cc1cccc(Cl)c1)C(=O)c1c[nH]nc1-c1ccc(OC(F)F)cc1', 'CN(c1ccc(F)cc1)S(=O)(=O)CC(=O)c1ccccc1', 'CN(c1cccc(N)c1)S(=O)(=O)c1cccc([N+](=O)[O-])c1', 'CN(c1ccccc1)S(=O)(=O)Cc1ccccc1N', 'CN(c1ccnc(Cl)c1)c1ccc(Cl)cc1OC[C@@H]1Cc2ccccc2O1', 'CNS(=O)(=O)CCNC(=O)N(C)Cc1ncc[nH]1', 'CNS(=O)(=O)c1ccc(Cl)c(C#N)c1Cl', 'CNS(=O)(=O)[C@@H](C)C(=O)NCc1ccccc1', 'CN(c1ccc(F)cc1)S(=O)(=O)c1ccc(NC(=O)[C@H]2CC(=O)N(c3ccccc3)C2)cc1', 'CN(c1ccccc1)S(=O)(=O)c1ccc([N-]S(=O)(=O)c2cccc(F)c2)cc1', 'CN(c1cccnc1)[C@H](C(=O)c1ccc(Cl)c(F)c1)C[C@H]1CCCO1', 'CNS(=O)(=O)[C@@H](C)Cc1ccc([N+](=O)[O-])cc1', 'CNS(=O)(=O)[C@@H]1CCN(Cc2cccc(OC)c2)C1', 'CNS(=O)(=O)CC(=O)N[C@H](C)c1ccc(OC)c(F)c1', 'CNS(=O)(=O)Cc1ccc(C(=O)N2CCC[C@@H](Nc3cc(C)ccc3C)C2)cc1', 'CNS(=O)(=O)CN(C)C(=O)C[C@@H]1COc2ccccc21', 'CNS(=O)(=O)C[C@H](C)ONC(=O)c1ccc2c(c1)CCO2', 'CN(CCCNC(=O)[C@@H]1COc2ccccc21)C1CC1', 'CN(CC(F)(F)c1ccc(Cl)cc1Cl)C(=O)N[C@@H]1CCC[C@H](C)C1', 'CN(CCc1cccc(Cl)c1)C(=O)c1cnc([C@@H]2CCCO2)nc1C', 'CN(CC1CC[NH+](Cc2ccon2)CC1)C(=O)c1c[nH]cn1', 'CN(CCOc1ccccc1)C(=O)CSc1nnc(SCC(N)=O)c(=S)c2ccccc12', 'CN(CC[NH+](C)C)C(=O)c1ccc2c(c1)N(C)c1ncccc1N=C1C=CC=C1C=C2', 'CNC(=O)c1ccc(CNC(=O)CCc2c[nH]cc2C(C)(C)C)cc1', 'CNC(=O)CCNC(=O)c1ccc(F)cc1F', 'CNC(=O)C(=O)NC[C@@](C)(O)c1ccc(F)cc1', 'CNC(=O)Cc1cc(=O)oc2ccc(Cl)cc12', 'CNC(=O)COc1c(Cl)cccc1NC(=O)N1CCSC(C)C1', 'CNC(=O)CNC(=O)C1CCN(c2ncnc3[nH]cnc23)CC1', 'CNC(=O)Cn1ccc(=O)n(C)c1=O', 'CNC(=O)CSc1nc(-c2ccccc2C)sc1C', 'CNC(=O)C[NH+]1CCN(c2ccccc2Br)CC1', 'CNS(=O)(=O)c1cc(Cl)c(C(=O)N2CCC[C@@H]2c2cccnc2)cc1Cl', 'CNc1cc[nH+]c(N2C[C@H]([NH3+])C(=O)CC2)c1', 'CNc1ccc(F)c(C[NH+]2CCCC2)c1', 'CNS(=O)(=O)c1cc(C)ccc1NC(=O)c1c(Cl)c(C)nn1C', 'CNS(=O)(=O)[C@@H](C)c1ccccc1', 'CNS(=O)(=O)[C@@H](NCc1cccc(OC)c1)c1ccccc1', 'CNS(=O)(=O)[C@@H](c1ccccc1)c1ccccc1Br', 'CNC(=O)NC(=S)N(CCCO)Cc1ccccc1', 'CNC(=O)N(Cc1ccccc1OC)[C@@H](C)c1nccn1C', 'CNC(=O)Nc1cccc(C(=O)N2CCCc3cc(C)ccc32)c1', 'CNC(=O)N1CCC(CCNC(=O)NCc2nc3cc(C)ccc3n2CC)CC1', 'CNC(=O)NNc1ccc(C(=O)NC(C)C)c(Cl)c1', 'CNC(=O)N[C@@H](C)c1ccc(F)cc1F', 'CNC(=O)N[C@H]1CC(=O)N2c3ccccc3C(=O)N2[C@H](c2ccccc2)[C@@H]1C(=O)c1cccc(Br)c1', 'CN=C(C#N)CCC(=O)N1C[C@H](C)OCC1(C)C', 'CN=C(c1ccccc1)c1ncn(-c2ccccc2)c1[N+](=O)[O-]', 'CN=C(OC)CC[NH+]1CCN(CCc2ccc(Cl)cc2)CC1', 'CN=C([C@@H]1COC1)c1ccccc1C(F)(F)F', 'CN=C(/C#N)c1cccc(C#N)c1', 'CNS(=O)(=O)c1c(C)[nH]c(C(=O)Cc2ccccc2)c1C', 'CNS(=O)(=O)c1ccc(F)cc1N[C@H]1CCC[C@@H](S(=O)(=O)N(C)C)C1', 'CNS(=O)(=O)c1c(C)c(C)nn1CC(=O)N1CCC[C@@H](C)CC1', 'CNS(=O)(=O)c1c(OCC)nn(C)c1C', 'CNS(=O)(=O)c1c(N=N/Nc2ccc(Cl)cc2C)nc2ccccc2n1', 'CNS(=O)(=O)c1c(F)cc(C(=O)NC2CCN(C(=O)COc3ccccc3)CC2)cc1C', 'CNS(=O)(=O)c1c(-c2ccccc2)nc(Nc2ccccc2)n1C', 'CNC(=O)[C@@H](NC(=O)c1cnn(-c2ccccc2)c1)c1nc2ccccc2cc1Cl', 'CNC(=O)[C@@H]1C[C@@H](O)C[NH+]1CC(=O)Nc1ccccc1F', 'CN[C@H](c1ccc(Cl)cc1Cl)c1cc(F)ccc1F', 'CN[C@H](NC(=O)c1ccccc1Cl)C(=O)NCCc1nccs1', 'CN[C@H](NS(=O)(=O)c1ccc(C)c(F)c1)c1ccc(F)cc1', 'CN[C@H](N[C@@H]1CCOC1(C)C)c1nnc2n1CCCCC2', 'CNC(=O)c1ccc(NC(=O)c2sc(C(=O)N3CCCCC3)cc2C)cc1', 'CNC(=O)c1csc(C(=O)O[C@@H]2CC[C@H](Sc3ccccc3)C2)c1', 'CNC(=O)c1ccc(NC(=O)CNC(=O)c2ccccc2Br)cn1', 'CNS(=O)(=O)Cc1ccc(F)c(C(=O)Nc2cccc(F)c2)c1OC', 'CNS(=O)(=O)[C@@H]1CCN(C(=O)[C@H]2CCCN2C(=O)Cc2ccccc2)C1', 'CNC(=O)[C@H](C)CN(C)C(=O)Nc1ccc(F)cc1Br', 'CNC(=O)[C@H]1CN(CC(=O)Nc2ccc(F)c(F)c2)CCO1', 'CNC(=O)c1cc([N+](=O)[O-])c(N2CCCC2)c2c1cnn2C[C@@H](C)CC', 'CNC(=O)c1ccc(OCCNC(=O)c2cnn(CC)c2)cc1', 'CNC(=O)c1cc[n+](CCC(=O)N[C@@H](C)CCOC)cc1', 'CNS(=O)(=O)CCNC(=O)NCc1cc(F)c(C)s1', 'CNS(=O)(=O)CCCNC(=O)/C=C/c1cccc2ccccc12', 'CNS(=O)(=O)CC(=O)N1CCN(C(=O)c2cccc(C)c2)CC1', 'CNS(=O)(=O)CCc1ccc(NC(=O)N2CCc3sccc3C2)cc1', 'CNS(=O)(=O)[C@@H]1CCN(C(=O)Nc2ccc(-c3ncccn3)cc2)C1', 'CN=C/C(=N\\OC[C@@H](O)c1ccc(Br)cc1)C1CC1', 'CN=C/S(=O)(=O)Cc1ccc(OC(C)C)cc1', 'CN=C/c1cc(-c2ccccc2)nc(S)c1C', 'CN=C/[NH2+]Cc1ccc(-c2ccccc2)[nH]c1=O', 'CNC(=O)[C@H]1CCC[NH+]1CC(C)(C)[NH2+]C', 'CNC(=O)[C@H]1[NH2+]CC[C@H]1C(=O)[O-]', 'CC[C@@H](CCO)NC(=O)N1C[C@@H](C)CCC[C@@H](C)C1', 'CC[C@@H](C(=O)NCCC1=CCCCC1)C(C)(C)C[NH+](C)C', 'CC[C@@H](C)OC(=O)c1nc(C)cc(C(N)=O)n1', 'CC[C@@H](COC)[NH+]1CC2(CC[NH+](Cc3ccccc3)CC2)C1', 'CC[C@@H](C#N)Oc1ccc(NC(=O)Cn2cc([N+](=O)[O-])cn2)cc1', 'CC[C@@H](C[C@H]1CCCC[NH+]1C)C(=O)[O-]', 'COc1nc2sn(-c3ccccc3)c(=O)c2c2c1CCCCCC2', 'COc1nc2snnc12', 'CNS(=O)(=O)[C@@H]1CCOCCN1C(=O)/C=C/c1ccc(Br)cc1', 'CNS(=O)(=O)[C@@H]1CCN(C(=O)c2ccccc2)C1', 'CNS(=O)(=O)[C@@H]1CC[NH+](C2CCN(Cc3cccc4ccccc34)CC2)C1', 'C[C@@H](c1nc(-c2cccnc2)n[nH]1)S(=O)(=O)c1ccc([N+](=O)[O-])cc1', 'C[C@@H](c1ncc(C(=O)[O-])n1C1CC1)c1occc1COC(F)(F)F', 'C[C@@H](c1nc2ccccc2[nH]1)C(=O)N1CCC(Cc2ccccc2)CC1', 'C[C@@H](c1ncoc1C1CC1)c1nnc(NC(=O)c2cccc(Cl)c2)s1', 'C[C@@H](c1ncnn1C)C(=O)[O-]', 'Cc1c(Br)oc(C)c1C(=O)NC(=S)NC(=O)COc1ccccc1', 'Cc1c(Br)cccc1N[C@H](C)C(=O)NNC(=O)c1cccs1', 'Cc1ccsc1CN1CCO[C@H](c2cccc([N+](=O)[O-])c2)C1', 'Cc1c([N+](=O)[O-])cc(C#N)c(C)c1SCC(=O)NCc1ccccc1', 'Cc1c([N+](=O)[O-])cccc1NC(=O)c1ccc(Cl)cc1', 'Cc1c([N+](=O)[O-])cc(C(C)(C)C)nc1[N+](=O)[O-]', 'Cc1ccsc1CNC(=O)[C@H]1CCC(=O)NC1=S', 'Cc1ccsc1OCC(=O)Nc1cccc(C(=O)N2CCOCC2)c1', 'Cc1ccsc1NC(=O)C1CCN(C(=O)[C@@H]2CCO[C@@H]2C)CC1', 'Cc1ccsc1[C@@H](N)Cc1ccc(Cl)c(F)c1', 'Cc1ccsc1-c1nnc(SCC(=O)[O-])s1', 'Cc1ccsc1S(=O)(=O)N[C@@H](C)CS(C)(=O)=O', 'Cc1ccsc1[C@H]1[NH+]=c2ccccc2=C1N', 'CNc1nn(Cc2ccc(Cl)cc2)c(N2CCCC2)c1/C=C/c1ccc(O)cc1', 'COC[C@@H]1OC(=O)c2cc(Br)ccc21', 'COC[C@@H]1[C@H](CC2CCCCC2)NC(C)=C1C(C)=O', 'C=C[C@@H]1N(C(=O)c2ccccc2)Cc2ccc(OC)cc21', 'C=C[C@@H]1N=C2O/N=N[C@H]2C(=O)[C@@H]1C', 'C=C[C@@H]1N[C@]2(C(=O)NCCN2c2ccccc2)c2ccccc21', 'O=C(c1cn(C[C@@H]2CCCO2)cc1O)[NH+]1CCC(CN2CCOCC2)CC1', 'Oc1cc(-n2nnnc2-c2ccccc2F)cc(-c2ccsc2)n1', 'Oc1cc(-n2nnnc2SCc2ccccc2)c2ccccc2n1', 'Oc1cc(-n2nnnc2Sc2ccccc2Cl)cs1', 'Oc1cc(-n2nnnc2S[C@@H]2CCCC[C@@H]2C[NH2+]Cc2ccccc2)ccc1Cl', 'Oc1cc(-n2nnnc2SSc2ccc(Cl)cc2[N+](=O)[O-])cc(Br)c1F', 'Oc1cc(C(F)(F)F)nc(CN(Cc2ccccc2)C2CC2)c1', 'Oc1cc(C(F)(F)F)cc2c1ncn2-c1ccc(F)cc1', 'Oc1cc(C(F)(F)F)cs1', 'Oc1cc(Cn2ncccc2=O)ccc1COC1CCN(C(=O)c2ccccc2)CC1', 'Oc1cc(Cn2nccc2C(=O)Sc2nc3ccccc3[nH]2)ccc1F', 'Oc1cc(COc2nc(-c3ccc(Cl)cc3)nc3c2CCC3)ccn1', 'Oc1cc(COc2ncccc2Cl)ccc1F', 'ClC(=S)NCC(=O)Nc1ccccc1F', 'ClC(=S)N\\c1ccc(F)c(-c2cscn2)n1', 'ClC(=S)N(Cc1ccccn1)[C@H]1C[C@@H]2CC[C@@H]1C2', 'ClC(=S)Nc1ccc(Sc2ncccc2Cl)cc1', 'ClC(=S)N1Cc2ccccc2S[C@@H]1[C@@H](O)C1CCOCC1', 'O=C(COc1cc([N+](=O)[O-])ccc1OCc1ccc([N+](=O)[O-])cc1)Nc1ccc([N+](=O)[O-])cc1', 'O=C(COc1cc(Br)c(Cl)cc1)NCc1cccc([N+](=O)[O-])c1', 'O=C(COc1cc(Cl)ccc1Cl)N1CCc2ccccc21', 'O=C(COc1cc([N+](=O)[O-])ccc1F)NCC(=O)N1CCOCC1', 'O=C(COc1cc(F)ccc1Cl)N1CCCCCC1', 'O=C(CNc1cnn(CN2CCCC2)c1)NCc1nc2ccccc2c2c1CCCC2', 'O=C(CNc1cnnc2cc(Cl)c(Cl)cc12)NCc1ccco1', 'O=C(c1c[nH]nc1CNCc1ccc(Br)cc1)N1CCN(c2cccc(Cl)c2)CC1', 'O=C(c1c[nH]nc1CN(Cc1ccccc1)C1CC1)Nc1ccc2c(c1)OCO2', 'O=C(c1c[nH]nc1CNc1ccc(-c2[nH+]c[nH]n2)cc1)N1CCOCC1', 'O=C(c1c[nH]nc1CN1C[C@H]2CCC[C@]21[NH2+]C1CCCCC1)N1CCC(C(=O)N2CCCCCC2)CC1', 'O=C(/C=C/Oc1ccccc1F)N[C@H]1[C@H](C[NH+]2CCCCC2)C[C@H]1c1ccc(F)cc1', 'COc1nnn(CC(=O)N2CCC[C@@H](O)C2)c1Cl', 'COc1nnn(C(=O)Cn2ncc3c(O)cc(Br)c(C)c3c2=O)c1C', 'COc1nnn(C)c1C[C@H](C)OC(=O)c1cc(C)n(C)n1', 'COc1nnn(CNC(=O)N2CCC[C@@H](C(N)=O)C2)c1N1CCCC1', 'CN[C@H](c1[nH+]cc(C)s1)c1ccc(Br)s1', 'CN[C@H](c1ccccc1)c1ccc(F)cc1Cl', 'CN[C@H](c1ncc(C(C)(C)C)cn1)c1ccc(Cl)cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])c(F)cc(Nc2cc(Cl)cc(Cl)c2)c1Nc1ccc([N+](=O)[O-])cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])c(Br)sc1[O-]', 'O=[N+]([O-])c1c([N+](=O)[O-])c(Br)ccc1OC(F)F', 'O=C(Cc1nonc1C(F)(F)F)C1CCN(S(=O)(=O)c2ccccc2)CC1', 'O=C(COc1ccc2ccccc2c1)NCc1ccc(Cl)s1', 'O=C(COc1ccc2nccnc2c1)N1CC[C@H]([N+]2c3ccccc3CS2)C1', 'O=C(CS(=O)(=O)Cc1ccccc1)Nc1cc(Oc2ccccc2)c(S(N)=O)cc1', 'O=C(Cc1c(Cl)cccc1Cl)N(c1ccccn1)Cc1ccccc1', 'O=C(Cc1c(F)cccc1F)c1ccc2c(c1)OCO2', 'O=C(Cc1cn2cccnc2n1)NCc1ccnc2ccccc12', 'O=C(Cc1c(O)cccc1O)N[C@@H]1CCCC[C@H]1c1ccccc1', 'O=C(Cc1nnnn1-c1ccccc1)N1CCO[C@H](c2ccccc2F)C1', 'O=C(C[NH+]1CCCC1)NC[C@@H](O)c1ccc(Cl)cc1', 'O=C(C[NH+]1CCC([C@H](Br)C2CC2)C1)N[C@@H]1CCCc2ccccc21', 'O=C(C[NH+]1CCC2(CC1)NCCN2C(=O)c1ccc2c(c1)OCC(=O)N2)OCCO', 'O=C(C[NH+]1CCCN(c2cc(F)cc(F)c2)CC1)Nc1ccccc1C(=O)NCc1ccccc1', 'O=C(C[NH+]1CCC[C@@H]1c1ccccc1)Nc1nnc(C2CCCCC2)s1', 'O=C(C[NH+]1CCC[C@H]1c1ccc(Cl)cc1)N1CCC[C@@H]1c1ccccc1', 'O=C(C[NH+]1CCC[NH+](Cc2cccc(Cl)c2)CC1)Nc1ccccc1F', 'O=C(C[NH+]1CCC[C@](O)(C[NH+]2CCCCC2)C1)N[C@@H]1CCOc2ccccc2C1', 'COc1ncc2nc(N/N=C/c3ccc(C#N)cc3)c(CC)nc2c1C', 'COc1ncc2n1CCC[C@@H]2[NH+]1C[C@@H]2CCCC[C@@H]21', 'COc1ncc2nn(C)c(=O)c(C=O)c2c1C', 'ClCc1nnsc1Cl', 'Clc1nnnn1c1c(Br)cccc1N1CCCCC1', 'Clc1nnnn1c1cccc(C(F)(F)F)c1', 'Clc1nnnn1-c1cn(Br)cn1', 'Clc1nnnn1-c1cn[nH]c1', 'Clc1nnnn1-c1cnccc1', 'Clc1nnnn1-c1cnn(C[C@@H]2CCCO2)c1', 'ClCc1nnsc1[N+](=O)[O-]', 'ClCc1nnsc1C(=O)NC1CCCCC1', 'ClCc1nnsc1[N-][C@@H]1CCS(=O)(=O)C1', 'ClCc1nnsc1NC1CCCCCC1', 'ClCc1nnsc1-c1cnn(-c2ccccc2)c1', 'ClCc1nnsc1Cl', 'ClCc1nnsc1Br', 'C[C@@H](C(=O)Nc1ccccc1F)[C@H](C)c1ccccc1', 'C[C@@H](C[NH+](C)CC(=O)NCc1cccc(F)c1)NS(C)(=O)=O', 'C[C@@H](C[NH+]1CCCCC1)N[C@H](C)c1ccc(CO)o1', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(-c2cn(-c3ccc(Cl)cc3Cl)[nH]c2=O)c1', 'Oc1cccc(-c2cncc(Cl)n2)c1O', 'Oc1cccc(-c2cnn(-c3ccc([N+](=O)[O-])c(NC4CCCCC4)c3)n2)c1', 'Oc1cccc(-c2cn[nH]c2)c1', 'Oc1cccc(Oc2cccc(COc3cnccn3)n2)c1', 'Oc1cccc(Oc2nc(-c3cccs3)nc3ccccc23)c1', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(O[C@@H]2CCO[C@]2(Cc2ccccc2Br)C2CC2)c1', 'Oc1cccc(O[C@@H]2[C@H](O)C(C(=O)[O-])CCS(=O)(=O)C2)c1', 'Oc1ccccc1OS(=O)(=O)c1ccccc1NC(=O)CC1CCCC1', 'Oc1ccccc1OSCC(=O)N1CCOC[C@@H]1Cc1ccccc1', 'Oc1ccccc1OSC(=S)N1N=C(c2ccc(Br)cc2)C(=O)[C@H]1c1ccccc1O', 'Oc1ccccc1OSCc1n[nH]c2nc([O-])oc12', 'Oc1ccccc1OSC1(CC2CC2)CCOCC1', 'Oc1ccccc1OSCOc1ccc(Br)cc1', 'Oc1ccccc1OSC[C@@H](O)c1ccccc1Cl', 'Oc1ccccc1OSC[C@H]([NH3+])CO', 'Oc1ccc(/C=C2/N=C(CN3CCN(c4ccccc4F)CC3)N=N2)cc1', 'Oc1ccc(/C=C2/SC(=S)N(c3ccccc3)C2=O)cc1', 'Oc1ccc(/C=C2/C(=O)N(c3ccccc3)SC2=S)cn1', 'Oc1ccc([N+](=O)[O-])c(I)c1OC[C@H](O)c1ccc(F)cc1', 'Oc1ccc([N+](=O)[O-])c(I)n1', 'Oc1ccc(/C=C/N=[N+]2CCC[C@H]2SC2CCCCC2)cc1', 'Oc1ccc(/C=C/N=C([O-])Oc2ccc([N+](=O)[O-])cc2)c(OCc2ccccc2)c1', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(S[C@@H](c2ccccc2F)N2CCC(c3c(F)cccc3F)CC2)c1', 'Oc1cccc(-c2c[nH]nc2CN2CCNC(=O)[C@H]2c2cccs2)c1', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(NCc2ccc(OCc3cccnc3)cc2)c1Cl', 'Oc1cccc(NCc2ncc(-c3ccccc3)o2)c1O', 'Oc1cccc(Br)c1O\n', 'Oc1cccc(N2CCCC2)c1', 'Oc1cccc(N2CCOCC2)c1', 'Oc1cccc(N2CCN(Cc3ccc(Cl)nc3Cl)CC2)n1', 'Oc1cccc(N2CC[C@H](NCc3ccc(F)cc3F)C2)c1', 'Oc1cccc(N2CC[NH+](Cc3ccc(Cl)o3)CC2)c1', 'Oc1cccc(N2CC[NH2+]CC2)c1O', 'Oc1cccc2cc(/C=N\\c3ccc(Cl)cn3)ccc12', 'Oc1cccc2cc(/C=C/c3ccccc3)ccc12', 'Oc1cccc2cc(/C=C(\\Cl)c3ccccc3)c(=O)c(=O)[nH]c12', 'Oc1cccc(Br)c1Oc1ccccc1F', 'Oc1cccc(S[C@@H]2CCC[NH2+]2)c1F', 'Oc1cccc(S[C@@H]2NC(=O)COc3ccc(Cl)cc32)c1', 'Oc1cc(F)ncc1C(=O)NNc1ncc(Cl)c(Cl)c1Cl', 'Oc1cc(F)ncc1COc1ccccc1F', 'Oc1cc(F)ncc1Cn1cnc2ccccc21', 'Oc1cc(F)ncc1CS[C@@H]1C=C[C@H]([NH3+])C1', 'Oc1cc(F)ncc1C[NH+](Cc1ccco1)Cc1ccco1', 'Oc1cc(F)ncc1C[NH2+]C[C@@H]1CN(Cc2ccccc2F)CCO1', 'O=[N+]([O-])c1c(Cl)nn(CNc2ccccc2Br)c1NC(=O)c1cccs1', 'O=[N+]([O-])c1c(Cl)nn(C[C@@H](O)COc2cccc(OCc3ccccc3)c2)c1-c1ccc(Cl)cc1', 'O=[N+]([O-])c1c(Cl)nn(C[C@H]2CCCO2)c1N1CCN(Cc2ccccc2)CC1', 'O=[N+]([O-])c1c(Cl)nn(-c2ccc([N+](=O)[O-])cc2)c1[O-]', 'O=[N+]([O-])c1c(OCc2ncon2)oc2ccc(Cl)cc12', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc([N+](=O)[O-])c1-c1cccs1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(C(F)(F)F)c1C#Cc1ccccc1F', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(Oc2ccc(Cl)cc2Br)c1C(F)(F)F', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(NCCC[NH+]2CCCCC2)c1Cl', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(I)c1-c1ccc([N+](=O)[O-])cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(F)c1C(=O)NCCCc1ccccc1Cl', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(-c2ccc(F)cc2)c1N', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(SCCCO)n1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(Cl)c1Oc1ccccc1Br', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(Br)c1N1c2ccccc2CC[C@@H]1Cc1c[nH]c2ccccc12', 'O=[N+]([O-])c1c(Cl)nsc1-c1ccccc1', 'O=[N+]([O-])c1c(Cl)nnc2c1COC2', 'O=[N+]([O-])c1c(Cl)nnc2onc(-c3cccc([N+](=O)[O-])c3)c12', 'O=[N+]([O-])c1c(Cl)nnc2nc(NCc3ccccc3)[nH]c(=O)c12', 'O=[N+]([O-])c1c(Cl)nnc2sc(-c3ccco3)nc12', 'O=[N+]([O-])c1c(Cl)nnc2[nH]nc(C3CCCCC3)c12', 'O=[N+]([O-])c1c(OCO)cccc1N1CCCC1', 'O=[N+]([O-])c1c(OCOc2ccc(Cl)cc2)ncnc1N[C@H](c1ccccc1)c1ccc(Cl)cc1', 'O=[N+]([O-])c1c(O)sc2c1CC[NH+](Cc1cccc(F)c1)C2', 'O=[N+]([O-])c1c(Br)cccc1-n1cc([O-])cn1', 'O=[N+]([O-])c1c(F)cc2n1CCC[C@@H]2COC1CC1', 'O=[N+]([O-])c1c(F)cc2n1[C@@H](c1ccccc2F)c(C2CC1)CN2/N=C=C1/N=NCCS1', 'O=[N+]([O-])c1ccncc1NCCC(=O)N1CCC[C@@H](c2nccs2)C1', 'O=[N+]([O-])c1ccncc1Nc1ccc([N+](=O)[O-])cc1', 'O=[N+]([O-])c1ccncc1N1C[C@H](O)CC[C@@H]1c1ccc(Cl)cc1', 'O=[N+]([O-])c1ccncc1N[C@@H]1c2ccccc2CC[C@@H]1O', 'O=[N+]([O-])c1ccncc1N[C@H]1CCN(c2ccccc2)C1=O', 'O=[N+]([O-])c1ccnc2ccc(F)c(-c3ncoc3C(F)(F)F)c12', 'O=[N+]([O-])c1ccnc2ccccc12', 'O=[N+]([O-])c1ccnc2cccnc12', 'O=[N+]([O-])c1ccnc2cccc(-c3nc4ccccc4s3)c12', 'O=[N+]([O-])c1ccnc2ccccc12', 'O=[N+]([O-])c1ccnc2cccnc12', 'O=[N+]([O-])c1ccnc2ccc([N+](=O)[O-])cc12', 'O=[N+]([O-])c1ccnc2ccc(C(=O)N3CCC[C@@H](O)C3)cc12', 'O=[N+]([O-])c1ccnc2ccc([N-]S(=O)(=O)c3cc(F)ccc3F)cc1c1ccccc12', 'O=[N+]([O-])c1ccnc2ccc(OC(=O)C3(c4ccccc4)CCC3)cc12', 'O=[N+]([O-])c1ccnc2ccc(NCCNS(=O)(=O)Cc3ccccc3)cc12', 'O=[N+]([O-])c1ccnc2ccc(F)cc12', 'O=[N+]([O-])c1ccnc2ccc(S(=O)(=O)C[C@H](O)c3cccs3)cc12', 'O=[N+]([O-])c1ccnc2ccc(Cl)c(F)c12', 'O=[N+]([O-])c1ccnc2ccc(Br)cc12', 'O=[N+]([O-])c1ccncc1[C@@H](O)c1ccccc1', 'O=[N+]([O-])c1ccncc1[C@@H]1CN(Cc2ccncc2)C1', 'O=[N+]([O-])c1ccncc1SC[C@@H]1C[C@H](O)c2ccccc2O1', 'O=[N+]([O-])c1ccncc1Sc1ccc(-n2cccn2)nc1', 'O=[N+]([O-])c1ccncc1S[C@@H]1CC(=O)N(c2ccccc2F)C1', 'O=[N+]([O-])c1ccn([C@H]2CCCN2S(=O)(=O)c2ccccc2Br)c1', 'O=[N+]([O-])c1ccn([C@H]2NC[C@@H](O)[C@@H]2O)c1', 'O=[N+]([O-])c1ccc2ncn([C@@H](CCl)c3ccccc3)c2c1', 'O=[N+]([O-])c1cccc2[nH]ccc12', 'O=[N+]([O-])c1cccc2nc(-c3cc(Cl)ccc3[N+](=O)[O-])[nH]c12', 'O=[N+]([O-])c1cccc2ncccc12', 'O=[N+]([O-])c1cccc2[nH]cn/c(=N/n3c4ccccc4nc3c1)OCOC2', 'O=[N+]([O-])c1cccc2[nH]cnc1c1cc(F)c(Cl)cc12', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn2c(=O)n(-c3ccccc3)nc12', 'O=[N+]([O-])c1c([N+](=O)[O-])cnnc1CSC1CCC1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1[N+](=O)[O-]', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1C[C@@H]1CCC[NH+](C2CCCC2)C1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1OC[C@@H]1CCCO1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1N[C@H]1CCCS1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1-c1ccc(Cl)cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1Sc1ccc(Cl)cc1Cl', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1Cl', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1[O-]', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1[C@H]1CCS(=O)(=O)C1', 'Oc1ccc([N+](=O)[O-])c(-c2csc(N3CCCCC3)n2)c1', 'Oc1ccc([N+](=O)[O-])c(Sc2cccc(Br)c2)c1', 'Oc1ccc([N+](=O)[O-])c(-n2ccnc2)c1C(F)(F)F', 'Oc1ccc([N+](=O)[O-])c(S[C@@H]2CC[NH+](Cc3ccccc3)C2)c1', 'Oc1ccc(F)c(F)c1C[NH+]1CCC(OCc2ccccc2)CC1', 'Oc1ccc(F)c(F)c1F', 'Oc1ccc(F)c(F)c1SCc1nc2ccccc2c(=O)[nH]1', 'Oc1ccc(F)c(F)c1Br', 'Oc1ccc(F)c(F)c1/N=N/OCc1ccc(F)cc1', 'Oc1ccc(/N=c2/[nH]cccc2CCNC(=O)c2cccnc2)cc1', 'Oc1ccc(/N=c2/[nH]ccnc2COc2ccccc2)cc1Cl', 'Oc1ccc(/N=c2/[nH]cc[nH]2)nc1', 'Oc1ccc(F)c(Sc2nc3c(cccc3c3C[NH+]4CCCCCC4)ccc23)c1', 'O=[N+]([O-])c1cc(C(=O)N[C@H]2CC(=O)N(S(=O)(=O)c3ccccc3)C2)ccc1Cl', 'O=[N+]([O-])c1cc(N2CCC(Oc3cc(F)cc(Cl)c3)CC2)ccc1OC[C@@H]1CCCO1', 'O=[N+]([O-])c1cc(N2CC(C(=O)NCc3cccs3)C2)ccc1Br', 'O=[N+]([O-])c1cc(N2CCOCC2)ccc1N1CCOCC1', 'O=[N+]([O-])c1cc(N2CCN(Cc3ccccc3N3CCOCC3)CC2)nc(-c2ccco2)n1', 'O=[N+]([O-])c1cc(N2CC[NH+](CC3=c4ccccc4=[NH+]C3)CC2)ncc1Cl', 'O=[N+]([O-])c1ccnc2cccnc12', 'O=[N+]([O-])c1ccnc2ccccc12', 'O=[N+]([O-])c1ccnc2cccnc1NC2', 'O=[N+]([O-])c1ccnc2cccnc12', 'O=[N+]([O-])c1cc(C(=O)O[C@H]2CCN(c3ncnc4sccc34)C2)cc([N+](=O)[O-])c1', 'O=[N+]([O-])c1cc(C(=O)N(Cc2ccco2)C[C@H]2CCOC2)on1', 'O=C(c1cc([S@](=O)Cc2cccs2)nc2ccccc12)c1ccc(F)cc1', 'O=[N+]([O-])c1cc(Cl)c2[nH]ncc2c1', 'O=[N+]([O-])c1cc(Cl)c(CN2CCO[C@@H](CO)[C@@H]2C[NH+]2CCCC2)cc1', 'O=[N+]([O-])c1cc(Cl)c(OCC(F)(F)F)cc1Br', 'O=[N+]([O-])c1cc(Cl)c(NCc2ccncc2)cc1F', 'O=[N+]([O-])c1cc(Cl)c(S(=O)(=O)N/N=C/c2ccc(Br)cc2F)cc1Cl', 'O=[N+]([O-])c1cc(Cl)c(Cl)cc1Oc1ccc2ccccc2c1', 'O=[N+]([O-])c1ccncc1F\n', 'O=[N+]([O-])c1ccncc1F\n\n', 'O=[N+]([O-])c1ccncc1[C@@H]1CSC(F)(F)C1=O', 'O=[N+]([O-])c1ccncc1[C@@H]1c2ccccc2C(=O)N1CN1C(=O)c2ccccc2C1=O', 'O=[N+]([O-])c1ccncc1[C@@H]1OCCN(c2ccc(OS(=O)(=O)c3ccccc3)nc2C2CC2)C1', 'O=[N+]([O-])c1ccncc1[C@@H]1SCCCCc2ccccc21', 'Oc1ccccc1-c1csc2nc(SCC(=O)OC[C@@H]3CCCO3)[nH]c12', 'Oc1ccccc1-c1c(C(=O)NCc2ccccn2)cc(-c2ccccc2)n1Cc1ccccc1', 'Oc1ccccc1-c1ccccc1COc1ccccc1Cl', 'Oc1ccccc1-c1c(C(=O)N(Cc2ccccc2)Cc2ccccc2)csc1C#N', 'Oc1ccccc1-c1c(O)ccc2c1CCCC2', 'Oc1ccccc1-c1c(F)cc(Cl)cc1F', 'Oc1ccccc1-c1c([C@@H](Cl)c2ccccc2)ncnc1Cl', 'Oc1ccccc1-c1c(-c2ccc3c(c2)OCCO3)n[nH]c1N1CCOC[C@@H]1C1CC1', 'Oc1ccccc1-c1c(Cl)cc(Cl)cc1CBr', 'Oc1ccccc1-c1c([O-])cc([N+](=O)[O-])c2noc(=O)n12', 'Oc1ccccc1[C@H](NCC1(CO)COC1)c1ccccc1', 'Oc1ccccc1[C@H](Nc1cccnc1OC[C@H]1CCCO1)c1ccc(Cl)cc1', 'Oc1ccccc1[C@H](N=C(CO)c1c[nH]c2ccccc12)c1cccs1', 'Oc1ccccc1[C@H](NNC(=O)c1ccc2nccnc2c1)c1ccccc1CO', 'Oc1ccccc1[C@H](N[C@H]1C[C@@H](O)[C@@H](O)c2ccccc21)c1nnc2cccnn12', 'Oc1ccccc1/N=N/n1c2ccccc2c2ncnn21', 'Oc1ccccc1/N=N\\c1ccccc1', 'Oc1ccccc1/C([O-])=N/S(=O)(=O)c1ccc(Br)cc1', 'Oc1ccccc1/N=N\\C(=N\\OCc1ccccc1)c1nc(-c2ccc(-c3ccccc3)cc2)no1', 'Oc1ccccc1/N=N\\c1ccccc1Br', 'Oc1ccccc1/N=N\\Nc1cc([N+](=O)[O-])ccc1C[NH2+]C1CC1', 'Oc1ccccc1/N=N\\n1cnnc1', 'Oc1ccccc1/N=N\\S(=O)(=O)CCNC(=O)Nc1ccc(I)cc1', 'Oc1ccccc1-n1nnnc1SCCCCC(=O)NCCc1ccccc1', 'Oc1ccccc1-n1nnnc1SCC(=O)N1CCc2ccccc21', 'Oc1ccccc1-n1nnnc1SCCCCCO', 'Oc1ccccc1-n1nnnc1SCc1ccc(Cl)c(Cl)c1', 'Oc1ccccc1CCC1CCN(Cc2ccccc2-n2cc[nH+]c2C)CC1', 'Oc1ccccc1CC(=O)N1CCOCCN1c1ccccc1', 'Oc1ccccc1CCOc1ccc([N+](=O)[O-])c(Cl)c1', 'Oc1ccccc1CCN1CC[NH+](Cc2cc(F)ccc2F)CC1', 'Oc1ccccc1CCn1nc(-c2cnccn2)ccc1=O', 'Oc1ccccc1CC[C@H](O)C[NH2+]Cc1cc(-c2ccccc2)no1', 'Oc1ccccc1CC[NH+]1CCN(Cc2ccccc2)CC1', 'Oc1ccccc1CC/[NH+]=N/CCC[NH+]1CCNN(C2CCOCC2)C(=O)C1', 'Oc1ccccc1CC[NH2+]Cc1ccc([N+](=O)[O-])cn1', 'Oc1cccc2nn[nH]c12', 'Oc1cccc2nn[nH]c1c1ccccc12', 'Oc1ccc([N+](=O)c2onc3c2CCCC3)c2ccccc12', 'Oc1ccc([N+](=O)c2onnc2Cn2cncn2)cc1', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(Br)c1O\n\n', 'Oc1cccc(Br)c1Oc1ccc(F)cc1Cl', 'Oc1cccc([C@H]2OC(/C=N/OC(=O)c3ccccc3)c3ccccc32)c1', 'Oc1cccc([C@H]2O[C@H](Br)[C@H](F)[C@H](O)C2)c1', 'Oc1cccc([C@H]2Oc3c(F)ccc(F)c3[C@H]3[NH2+]CCC[C@H]32)c1', 'Oc1cccc([C@H]2O[C@@H](c3ccccc3)S/C2=N\\C2=C/C=C\\c3nc4ccccc4n3-2)c1', 'OCS/C=C/c1cnc(C2CCCCC2)nc1Cl', 'OCSc1ccccc1C(=O)NCc1ccccc1Oc1ccccc1', 'OCSc1nc2ccccc2[nH]1', 'OCS1(=O)CCN(C(=O)COc2ccccc2)CC1', 'OCS/C=C/Nc1cc([N+](=O)[O-])ccc1N1CCCCC1', 'OCS/C(=C/C=C/O)C1(c2ccc([N+](=O)[O-])cc2)CC1', 'OCSc1ccc2cc(Cl)ccc2c1', 'OCSc1cn[nH]c1C#N', 'OCS1(=O)CCN(C(=O)c2ccc(F)cc2F)CC1', 'OCS1(Cc2ccccc2)CC[NH2+]CC1', 'OCc1nc(C(F)(F)F)cs1', 'OCc1nc(NCc2ccc(Cl)cc2)ncn1', 'OCc1nc(-c2ccccc2Br)cs1', 'OCc1nc(Cl)cc(C2CC2)n1', 'OCc1nc([O-])c(-c2ccccc2)o1', 'OCc1nc([C@H]2CCO[C@H]2c2ccccc2)no1', 'OCc1n[nH]cc1OCC1CC[NH+](Cc2ccccc2)CC1', 'OCc1n[nH]cc1Oc1ccc(Br)cc1', 'OCc1n[nH]cc1O[C@@H](c1ccccc1)[NH2+]CCN1CCCC1', 'OCc1n[nH]cc1O[C@H](CN1CC[NH2+]CC1)[C@H](Cc1ccccc1)C(F)F', 'OC[C@@]1(c2ccccc2)N[C@H](c2cccc(F)c2)C(=O)N1COc1ccccc1', 'OC[C@@]1([NH3+])Oc2ccccc21', 'OC[C@@]1([NH3+])CCc2ccccc21', 'O=C(c1cn[nH]n1)N([N+]1CCC1)[C@@H]1CCSc2ccccc21', 'O=C(c1cn[nH]n1)N(Cc1ccsc1)c1nc(C(F)(F)F)cs1', 'O=C(c1cn[nH]n1)N(c1nncs1)c1ncccn1', 'Oc1cc(-n2nc(Cl)cc2C(=O)c2cc(Cl)ccc2Cl)ccc1O', 'Oc1cc(-n2nc([C@@H](Cl)C(=O)[O-])ccc2=O)co1', 'CN(c1ccn(Cc2ccccc2)n1)C1CCCC1', 'CN(c1ccn(C)c1)CNC(=O)CCc1c(O)cccc1F', 'CN(c1ccn([C@@H]2CCS(=O)(=O)C2)c1)c1nc(C2CC2)no1', 'CN(c1ccn(-c2ccccc2Cl)n1)C1CCCC1', 'CN(c1ccn([C@H]2CCCCN2C(=O)C(=O)Nc2cccc(Br)c2)c1)C(C)C', 'Cc1c(Br)oc2nc(CCCl)ncc12', 'Cc1c(Br)onc1CCN(C)C(=O)NCc1ccccc1', 'Cc1cnc2oc(C)c(C(=O)N[C@H]3CC(=O)N(c4ccccc4F)C3)c2c1', 'Cc1cnc2[nH]c(C(=O)Nc3ccc4c(c3)OCO4)cc2c1', 'Cc1cnc2c(c1)[nH]c(=N)n2C[C@@H]1CCCO1', 'Cc1cnc2ncc(C(=O)OCC#C/C=C/c3ccc(C#N)c(Br)c3)cc2c1', 'Cc1sn(-c2ccc(F)cc2)c(=S)c1/C=N/NC(N)=O', 'Cc1c([C@@H](C)NC(=O)c2cc(C)n(-c3ccc(Cl)cc3)c2C)c(=O)n(-c2ccccc2)[nH]1', 'Cc1c([C@@H]2CCC[NH2+][C@@H]2C)cnn1C', 'CN=N[C@]1(C#N)CCC[C@@H]([NH+]2CCN(c3ccccc3F)CC2)C1', 'Oc1cc(Cn2c[nH+]c(-c3ccccc3)n2)ccc1Cl', 'O=C(/N=C/N=C/c1cc2ccccc2o1)c1ccc(Br)cc1', 'O=C(/N=C/c1ccc([N+](=O)[O-])cc1)C(F)(F)c1ccccc1', 'O=C(/N=C/c1ccccc1)c1cccc(C(=O)N2CCOc3ccccc3C2)c1', 'O=C(/N=C/c1ccc2c(c1)N(CCc1ccccc1)c1ccccc1-2)c1ccc[nH]1', 'O=C(/N=C/c1ccco1)c1ccnc(N2CCCCC2)c1', 'O=C(/N=C/c1cccnc1)c1nc2cc(Cl)ccc2o1', 'O=C(/N=C/c1ccc[nH]1)Nc1ccc(-c2ccc(F)cc2)nc1', 'O=C(/N=C/NCCC[NH+]1CCCCCCC1)N[C@H]1CCCOc2ccccc21', 'O=C(/N=C/NC(=S)N1CCCC1)Sc1cccc(I)c1', 'O=C(/N=C/NCc1ccccc1)c1ccc(F)cn1', 'O=C(/N=C/NC[C@](O)(c1cccc(Br)c1)c1ccccc1)c1cccc([N+](=O)[O-])n1', 'Oc1cc(C[NH2+][C@@H]2CCCc3ccccc32)c(F)cc1Br', 'Oc1ccc(/C=N/S[C@H]2CCS(=O)(=O)C2)cn1', 'Oc1ccc(/C=N/S(=O)(=O)c2ccc([N+](=O)[O-])cc2)cc1', 'Oc1ccc(/C=N/Sc2nnc3c(-c4ccc(F)cc4)nnn3c2[O-])cc1', 'Clc1nncnc1[N-]C(=O)c1ccc(C#N)cc1', 'Clc1nncnc1[N-]c1ccc(OCc2ccccn2)cc1', 'Clc1nncnc1[N-]OCC(=O)c1cccs1', 'Clc1nncnc1[N-]N1CCN(C(=O)C2CCCCC2)CC1', 'Clc1nncnc1[N-][C@@H]1CCC[NH+](CC(F)(F)F)C1', 'Clc1nncnc1[N-]S(=O)(=O)c1cc(F)cc(F)c1', 'Clc1nncnc1[N-][C@H]1CC[C@@H]2CCCC[C@@H]21', 'O=C(c1ccc(OCC(=O)N2CCCCC[C@@H]2C(=O)[O-])cc1)N1CCN(c2ncccn2)CC1', 'O=C(c1ccc(O)c(Cl)c1)N1CCC([NH+](Cc2ccco2)C2CC2)CC1', 'O=C(c1ccc(OS(=O)(=O)c2ccco2)cc1)[S@@](=O)c1ccc(Cl)cc1', 'O=C(c1ccc(O[C@H]2CCOC2)cc1)N1CCCC1', 'COC[C@H]1CCC[NH+]([C@@H](C)c2[nH+]ccn2CC)C1', 'COC[C@H]1[NH2+][C@@H](c2ccccc2OCc2cccc(C)c2)CC1', 'Oc1ccc([N+](=O)Cc2nc(C3CC3)no2)c2ccccc12', 'Oc1ccc([N+](=O)Cc2noc3ccccc23)c2ncccc12', 'Oc1ccc([N+](=O)Cc2n[nH]c(Cn3c(=O)[nH]c(=O)n3C)c2C(F)(F)F)cc1', 'Oc1ccc([N+](=O)Cc2nc3ccccc3[nH]2)c2c1CCCC2', 'Oc1ccc([N+](=O)Cc2nnnn2-c2ccccc2)c(Cl)c1', 'Oc1ccc(Oc2ccc(C=O)cc2)cc1', 'Oc1ccc(Oc2ccc(NCc3nc4ccccc4[nH]3)cc2)c(Cl)c1', 'Oc1ccc(Oc2ccc(F)cc2)cc1CN1CCN(Cc2ccccc2)CC1', 'Oc1ccc(Oc2ccc(-c3noc4cc(F)ccc34)nc2)cc1', 'Oc1ccc(Oc2ccc([C@H]3CCCN(c4ncccn4)C3)nn2)cc1', 'Oc1ccc(Oc2ccc(Br)cc2)cc1O', 'Oc1ccc(Oc2ccc(/C=N/NC[C@@H](c3ccccc3)c3ccco3)nc2Cl)cc1', 'Oc1ccc(F)cc1-n1c(=S)[nH]c2cc(Cl)ccc21', 'Oc1ccc(F)cc1-n1cccc1C(=O)NCc1ccccc1Br', 'Oc1ccc(F)cc1-n1cnc2ccc(Nc3ccccc3)cc2c1=O', 'Oc1ccc(F)cc1OCCN1CCN(c2cc[nH+]cc2)CC1', 'Oc1ccc(F)cc1OCc1ccn(-c2cccc(F)c2)n1', 'Oc1ccc(F)cc1OC[C@H](O)c1ccc(Cl)cc1', 'Oc1ccc(F)cc1OC[C@@H]1Cc2ccccc2S1', 'Oc1ccc(F)cc1NS(=O)(=O)c1ccc(Cl)cc1', 'Oc1ccc(F)cc1NSc1ccc2nnc(-c3ccccc3)n2c1', 'Oc1ccc(F)cc1-c1onc(N2CCCCCC2)n1', 'Oc1ccc(F)cc1-c1ccccc1[C@@H]1CCCN(C(=O)c2cnnc3c2s3)C1', 'Oc1ccc(F)cc1-c1nnc(CNCc2ccc(C(F)(F)F)cc2F)o1', 'Oc1ccc([N+](=O)[O-])c(Cl)n1', 'Oc1ccc(Oc2cc([C@@H](O)C[NH+]3CCCC3)ncn2)cc1F', 'Oc1ccc([N+](=O)c2cc(CCO)c(C(=O)[O-])cc2Br)cc1', 'Oc1ccc([N+](=O)c2cc(C(=O)NCCc3c[nH]c4ccccc34)ccc2F)cn1', 'Oc1ccc([N+](=O)c2cc(Cc3ccc(F)cc3)cs2)cc1', 'Oc1ccc([N+](=O)c2cc(COC3CCCC3)c[nH]2)cc1', 'Oc1ccc([N+](=O)c2cc(C#N)cc(Cl)c2)cc1O', 'Oc1ccc(/N=c2/[nH]c3cc(Br)c([N+](=O)[O-])cc3s2)cc1O', 'Oc1ccccc1-n1nnnc1SCc1ccc(Cl)cc1Cl', 'Oc1ccccc1-n1nnnc1COc1ccc2nccnc2c1', 'Oc1ccccc1-c1noc([C@H]2CCOC3(CCCC3)C2)n1', 'Oc1ccccc1-c1n[nH]c(Nc2cc(F)c(Cl)cc2Cl)n1', 'Oc1ccccc1-c1nc(-c2csc(NC3CCC3)c2)no1', 'Oc1ccccc1-c1nnn(CCc2ccc(F)cc2)c1-c1ccccc1', 'Oc1ccccc1[C@H]1OCCN1CC[NH+]1CCCCC1', 'Oc1ccccc1[C@H]1Oc2ccccc2Oc2ccccc21', 'Oc1ccccc1[C@H]1ON=C(/C=N\\c2ccccc2Cl)[C@@H]1C(=O)[O-]', 'Oc1ccccc1[C@H]1OP(=O)OCCCN1C(=O)c1ccccc1', 'Oc1ccccc1[C@H]1O[C@@H](CO)[C@@H](O)[C@H](O)[C@@H]1O', 'O=[N+]([O-])c1c(OC[C@H](O)COc2cccc(F)c2)nccn1', 'O=[N+]([O-])c1c(OC[C@H]2CCCO2)[nH]c2ccc(Cl)cc12', 'O=[N+]([O-])c1c(Br)cc(I)cc1O[C@@H]1C=CS(=O)(=O)C1', 'O=[N+]([O-])c1c(Br)cc(F)cc1C(=O)Nc1cc(C(F)(F)F)ccc1Cl', 'O=[N+]([O-])c1c(Br)cc(S(=O)(=O)N2CCOCC2)cc1OC(F)F', 'O=[N+]([O-])c1c(Br)cc(Cl)cc1C(=O)N1CCOCC1', 'O=[N+]([O-])c1c(Br)cc(Br)cc1F', 'O=[N+]([O-])c1c([O-])sc(OC2CCCCC2)c1[N+](=O)[O-]', 'O=[N+]([O-])c1c(Cl)nsc1NC[C@H](O)c1ccccc1Cl', 'O=[N+]([O-])c1c(Cl)nsc1Nc1ccc(Cl)cc1F', 'O=[N+]([O-])c1c(Cl)nsc1N1CC[NH2+]CC1', 'O=[N+]([O-])c1c(Cl)nsc1N[C@@H]1CC[NH+]2CCCC[C@@H]12', 'O=[N+]([O-])c1c(Cl)nsc1N[C@H]1C[C@@H]2C(=O)CCN(Cc3ccccc3)C[C@H]21', 'O=[N+]([O-])c1c(Cl)nsc1N/N=C\\c1ccc(-c2cccnc2Cl)cc1', 'N=CCNC(=O)C1CCN(C(=O)CNC(=O)c2ccc(F)cc2)CC1', 'N=CCNc1c(-c2[nH+]c3ccccc3[nH]2)c(=O)oc2ccccc12', 'N=CCN1CC(=O)NC(=S)N1C[C@H]1C=CCC1', 'N=CCNS(=O)(=O)c1cccc(C(=O)N[C@@H]2CCCC[C@@H]2C)c1', 'OCS(=O)(=O)c1ccc(N2CCN(c3ccccc3)CC2)cc1', 'OCS(=O)(=O)c1c(Cl)cccc1Cl', 'OCS[C@@H](Cc1ccccc1Cl)c1c(F)cnc(F)c1F', 'OCS[C@@H](c1ccncc1)c1ccc(Cl)cc1', 'OCS[C@@H]([NH2+]C[C@@H]1CCCO1)c1nccn1C1CCCC1', 'OCc1ncsc1Nc1ccccc1C(=O)N1CCCC1', 'OCc1nc[nH]n1', 'Oc1cc(-n2cc(Br)cn2)ccc1C(F)(F)F', 'Oc1cc(-n2cc(-c3ccccc3)nc2-c2ccc(Br)cc2F)ccc1F', 'Oc1cc(-n2cc(F)cc2Cn2cc[nH+]c2)ccc1Nc1ccc(F)c(F)c1', 'Oc1cc(-n2cc(C(F)(F)F)ccc2=O)ccc1NC[C@H]1CCCC[C@@H]1O', 'Oc1cc(-n2cc(Cn3cncn3)c(Cl)cc2=O)cc(OCC(F)F)c1', 'Oc1cc(-n2cc(C[NH+]3CCN(c4ccccc4)C3=O)CC2)ccc1Br', 'Oc1cc(O)cc2[nH]cnc12', 'Oc1cc(O)cc2ncnc(N[C@@H](CO)[C@H](O)C(F)(F)F)c12', 'N[C@@H](Sc1ccc(Cl)s1)C(=O)OCC1CC1', 'N[C@@H](Sc1nnc2ccccn12)[C@@H]1CCCO1', 'Cc1snnc1SCC(=O)Nc1cccc([N-]S(C)(=O)=O)c1', 'Cc1snnc1S(=O)(=O)c1ccc(S(=O)(=O)N(C)C)cc1', 'Cc1snnc1Sc1nc(N)c2c(C)cccc2n1', 'Cc1snnc1S[C@@H](C)C(=O)N[C@@H](c1ccc(F)cc1)C1CC1', 'Cc1snnc1S[C@H](C)C(=O)N(C)Cc1ccc(=O)[nH]c1', 'O=C(c1c[nH]cn1)[C@H](Nc1ccsc1)c1ccccc1', 'O=C(c1c[nH]cn1)[C@H]1CCCN1C(=O)Cc1ccco1', 'O=C(c1c[nH]c([O-])n1)N1CCN(S(=O)(=O)c2ccc3c(c2)CCCC3)CC1', 'O=C(c1c[nH]cn1)[C@H]1CC(=O)N(Cc2ccc(Cl)cc2)C1', 'O=C(c1c[nH]cn1)[C@H]1Oc2ccccc2NC1=O', 'O=C(c1c[nH]cn1)[C@H]1[C@@H](C2=CC=CC=SC2)N=NN1c1ccccc1Cl', 'O=C(c1c[nH]cn1)[C@H]1[C@H](c2cccc(Cl)c2)CCC1=O', 'Nc1[nH]nc(CCC(=O)N[C@@H]2CCCC[C@@H]2Cc2ccccc2)n1', 'Nc1[nH]nc(C2CC2)c1Br', 'Nc1[nH]nc(COc2cccc(Br)c2)c1N', 'Nc1[nH]nc(CNc2ccc3c(c2)CCCC3)c1[N+](=O)[O-]', 'Nc1[nH]nc(Cn2cc(-c3ccc(Cl)cc3)cn2)n1', 'Nc1[nH]ncc1CCC(=O)Nc1ccn(-c2ccccc2)n1', 'Nc1[nH]ncc1C[S@](=O)Cc1ccccn1', 'Nc1[nH]ncc1C(=O)O[C@@H]1CCCC[C@H]1c1ccccc1', 'Nc1[nH]ncc1COc1ccc(Cl)cc1', 'Nc1[nH]ncc1CN(Cc1ccccc1[N+](=O)[O-])C[C@H]1CCCO1', 'Nc1[nH]ncc1C[NH+](Cc1ccsc1)C[C@@H]1CCC[NH+](C2CCCCC2)C1', 'Nc1[nH]ncc1C[NH3+]', 'NS(=O)(=O)Cc1cn(Cc2ccccc2)nc1SCc1ccccc1', 'NS(=O)(=O)Cc1cnn(-c2ccccc2)c1', 'NS(=O)(=O)Cc1cncc([N+](=O)[O-])c1', 'NS(=O)(=O)Cc1cn2ccsc2n1', 'NS(=O)(=O)CC[C@@H]([NH2+]Cc1ccccc1)c1ccccn1', 'NS(=O)(=O)CC[C@@H]1N=Nc2ccccc21', 'NS(=O)(=O)/C(CNC(=O)CCc1ccccc1)[C@H]1CCCC[C@@H]1C(=O)[O-]', 'N=C1[C@H](NCSc2ccc(F)cc2)C(=O)N(c2ccccc2)N1CCc1ccc(Cl)cc1', 'N=C1[C@H]2CC3=CC=CC(C)(C)CC[C@@H]3C[C@@H](SCC(=O)Nc3ccccc3F)[C@@H]12', 'Oc1ccc(F)c([C@@H]2O[C@@H]3CCC[C@H]2[C@@H](O)[C@@H]3O)c1', 'Oc1ccc(F)c([C@@H]2CCC[NH+]2Cc2ccccc2CO)c1', 'CNS(=O)(=O)C[C@H](C)[NH2+]Cc1ccc(Cl)c2ccc(F)cc12', 'CNS(=O)(=O)C[C@H]1CCCN1C(=O)CCc1ccccc1OC', 'CN[C@H](CCc1[nH+]ccs1)c1ccsc1', 'CN[C@H](C(=O)n1ccc(=O)[nH]c1=O)c1ccc(C)c(C)c1', 'CN[C@H](C)c1ccc(Cl)cc1', 'CN[C@H](Cc1ccc(C)cc1)c1ccc(C)o1', 'CN[C@H](C1(CCC2CC2)CC1)c1ccc(C)cc1', 'CN[C@H](CN1CC[NH2+]CC1)Cn1cc[nH+]c1C', 'CN[C@H](Cn1cncn1)c1ccncc1', 'CN[C@H](C#N)c1cc(F)ccc1SCc1ccc(Cl)cc1', 'CN[C@H](CSC)c1ccncc1', 'CN[C@H](C[C@]1(O)CCSC1)C(=O)[O-]', 'O=C(/C=C/c1ccno1)N[C@@H](c1ccccc1)c1ccccn1', 'O=C(/C=C/c1ccn[nH]1)O[C@@H]1CCC[C@H](C(F)(F)F)C1', 'O=C(/C=C/c1ccnc2c1-c1ccccc1O2)N1CCC(Cc2ccccc2)CC1', 'O=C(/C=C/O[C@@H](O)c1ccc(F)cc1)NCc1ccccc1', 'O=C(/C=C/O[C@@H]1CCS(=O)(=O)C1)Nc1ccccc1C(=O)N1C[C@@H](C)CC[C@H]1C(=O)NCC(F)(F)F', 'O=C(/C=C\\Nc1cccc(Br)c1)c1ccc(F)cc1F', 'NS(=O)(=O)Cc1noc(S(=O)(=O)N2CCN(c3ccccc3Cl)CC2)n1', 'NS(=O)(=O)Cc1nnc2ccc(Br)cn12', 'NS(=O)(=O)Cc1ccc(NC(=O)[C@H](O)c2cccc(Cl)c2)cc1', 'NS(=O)(=O)Cc1cccc(NC(=O)c2cnn(-c3ccccc3)c2)c1', 'NS(=O)(=O)Cc1noc(Cn2c(=O)ccc3nc(C(=O)[O-])cn32)n1', 'NS(=O)(=O)Cc1noc(I)c1[N+](=O)[O-]', 'NS(=O)(=O)Cc1noc([C@@H]2CCCN(c3ncccn3)C2)n1', 'NS(=O)(=O)Cc1noc(-c2sncc2-c2ccccc2)n1', 'NS(=O)(=O)Cc1noc(S(=O)(=O)NNC(=O)C(c2ccccc2)c2nc[nH]n2)n1', 'Nc1cn[nH]c(=O)c1[N-]S(=O)(=O)c1ccc([N+](=O)[O-])cc1', 'O=C(C1=c2[nH]ccc2=[NH+]C1)N1CC[C@@H](Nc2cccc(Cl)c2)C1', 'O=C(C1=c2ccccc2=[NH+][C@@H]1C(F)(F)F)N1CCN(Cc2ccc([N+](=O)[O-])cc2)CC1', 'ClC(c1cc(C2CC2)nn1-c1ccc(Br)cc1)c1ccc(Cl)c([N+](=O)[O-])c1', 'ClC(c1cc(F)ccc1Cl)c1ccc2c(c1)CCO2', 'ClC(c1cc(Cl)ccc1Cl)N1CCc2ccc(F)cc2C1', 'ClC(c1cc(Br)ccc1F)c1ccc2c(c1)OCO2', 'ClC(/N/N=C/c1ccccc1)C1CCOCC1', 'ClC(/N/c1c(Cl)cccc1Cl)(SCc1c[nH]c(=O)[nH]1)Cc1ccccc1', 'ClC(c1cccs1)c1cc(N2CCN(C(=O)Cc3ccccc3)O2)ccn1', 'ClC(c1ccc(F)cc1)[C@@H](O)c1cc(Br)cc(F)c1', 'ClC(c1ccccc1)[C@@H]1CCS(=O)(=O)C1', 'ClC(c1ccc2c(c1)CCC[C@H]2[NH2+]C[C@@H]1CCCO1)c1ccc(F)cc1', 'N[C@@H](CCC(F)(F)F)C(=O)N1CC[NH+]2CCCC[C@@H]2C1', 'N[C@@H](CC(=O)N1CCCC1)c1nc2cc(F)ccc2o1', 'N[C@@H](CCc1nccs1)[C@H](c1ccccc1)[n+]1ccccc1', 'N[C@@H](CC1CCOCC1)C(=O)Nc1ccsc1', 'N[C@@H](CCOc1cccc(C(F)(F)F)c1)N(C)Cc1ccc(-n2cncn2)cc1', 'N[C@@H](CCSc1ccncc1)c1ccc(Cc2ccccc2)s1', 'N[C@@H](CC[NH+]1CCN(c2ccccc2)CC1)c1ccc(Cl)cc1', 'O=C(Cc1noc(-c2ccncc2)n1)N[C@@H]1CCS(=O)(=O)C1', 'O=C(Cc1noc(C2CC2)n1)[C@@H]1CCCCN1S(=O)(=O)c1ccccc1', 'O=C([O-])/C(=C\\Cc1ccccc1)n1nnc2ccccc21', 'O=C([O-])/C=C(/N=C\\c1[nH]nc(C(F)(F)F)c1Cl)NCc1ccccc1', 'O=C([O-])/C=C(\\Nc1ccc(Br)cc1)N[C@@H]1CCOC1', 'O=C(CS(=O)(=O)CCC1CCCCC1)NNc1ccc2c(c1)COC2', 'O=C(CS(=O)(=O)CCc1ccccc1)NCc1ccccc1F', 'O=C(CS(=O)(=O)CC1CCC1)NCC#N', 'O=C(CS(=O)(=O)CCN1CCC([NH+](CC)C2CC2)CC1)c1ccc(F)cc1F', 'O=C(CS(=O)(=O)CC[C@H](O)c1ccccc1)N(Cc1ccc(F)cc1)C1CC1', 'O=C(CCOc1ccc(Cl)cc1)N(Cc1cccc(F)c1)C(=O)c1nc2ccccc2c(=O)[nH]1', 'O=C(CCS)Nc1ccccc1C(=O)Nc1ccccc1C(F)(F)F', 'O=C(CCS)/C(=N\\c1ccccc1)Sc1nncn1CC(=O)[C@H](O)c1cccc(Br)c1', 'O=C(CCS)C1=Cc2ccccc2OC1', 'O=C(CCS)c1ccccc1', 'O=C(Cc1conn1)Nc1ccc2ccccc2c1', 'O=C(Cc1cnc(-c2ccncc2)c(Cl)c1)NC(c1ccccc1)c1ccccc1', 'O=C(Cc1cnccn1)Nc1cc(Cl)c(Cl)cc1Cl', 'O=C(Cc1cnc2ccccc2n1)Nc1ccc(N2CCCC2=O)c(Cl)c1', 'O=C(Cc1cnco1)Nc1cccc2c1CCCC2', 'O=C(Cc1cnc[nH]1)NNC(=O)c1cccs1', 'O=C(Cc1c(Cl)cccc1Cl)Nc1nnn[n-]1', 'O=C(Cc1c(Cl)n[nH]c1Cl)NCc1cccc(Cl)c1', 'O=C(COc1cc(Br)ccc1Br)Nc1ccc(Cl)cc1', 'O=C(COc1cc(Cl)c(Cl)cc1Cl)c1ccc(C(F)(F)F)cc1', 'O=C(COc1cc(Cl)ccc1N1CCC(=O)[C@@H]1c1ccccn1)N1CCOCC1', 'O=C(COc1cc(Cl)c2c(c1)OCCO2)Oc1ccc(-c2ccccc2)cc1', 'O=C(COc1cccc(C(F)(F)F)c1)N1CCC[C@H]1c1ccccn1', 'O=C(COc1ccccc1)Nc1ccccc1N1CCCCC1=O', 'O=C(COc1ccccn1)N[C@H](Cc1ccccc1)n1cncn1', 'O=C(COc1cc([N+](=O)[O-])ccc1[N+](=O)[O-])N1CCCC1', 'O=C(COc1ccc2c(c1)OCOC2)Nc1cccc(F)c1Cl', 'O=C(COc1ccc2cc([N+](=O)[O-])ccc2c1)Nc1cccc(Cl)c1', 'O=C(COc1cc(Br)c(O)cc1)N1CCN(S(=O)(=O)c2ccc(Br)cc2)CC1', 'O=C(COc1cc(Br)c2ccccc2n1)Nc1ccccc1', 'O=C(COc1cc(Br)ccc1OCc1ccccc1)N(Cc1ccc(F)cc1)S(=O)(=O)c1ccco1', 'O=C(COc1cc(Br)cnc1Cl)N(Cc1ccc(F)cc1)C1CC1', 'O=C(COc1ccc2nsnc2c1)N[C@@H]1CCCC[C@H](O)C1', 'O=C(COc1ccc2ncccc2c1)Nc1ccc2nc(-c3ccc(Cl)s3)[nH]c(=O)c2c1', 'O=C(COc1ccc2nnnn2c1)NCc1cccc(C[NH+]2CCCC2)c1', 'O=C(CNc1nn(CC[NH+]2CCCCCC2)c2c1CCCC2)Nc1ccccc1', 'O=C(CNc1nn(Cc2ccncc2)nc1Cl)Nc1cccc2c1C[C@@H](O)C2', 'O=C(CNc1nn(C2CC2)c(=O)c2ccccc12)NCc1csc(Nc2ncccn2)n1', 'O=C(CCOn1c2ccccc2c2ccccc21)Nc1ccc2c(c1)CCCC2', 'O=C(CCc1cc[nH+]cc1)Nc1cccc(CNc2ncccn2)c1', 'O=C(CCc1cc(-c2ccccc2)n[nH]1)N1CCCCCCC1', 'O=C(CCc1ccc([N+](=O)[O-])cc1)N1CCN(C(=O)c2sc3ccc(F)cc3c2C)CC1', 'O=C(CCc1cc2ccccc2oc1=O)OCc1cccnc1', 'O=C(CCc1ccncc1)N/N=C/c1ccc(-c2ccccc2F)o1', 'O=C(CCc1ccsc1)Nc1ccc2nc(C3CC3)oc2c1', 'O=C(CCc1cc[nH]n1)Nc1ccc(Cl)cc1O', 'O=C(Cc1nn[nH]n1)c1ccc2c(c1)Cc1ccccc1C2', 'O=C(Cc1nn(CC[NH+]2CCCCC2)c(=O)c2ccccc12)N1CCOCC1', 'O=C(Cc1nn(Cc2ccccc2)c2c1CCC2)N1CCCOCC1', 'O=C(Cc1nn(CN2CCCCC2)c2c1CCC2)N1CCC[C@H](Oc2ccccc2Cl)C1', 'O=C(Cc1nn(C[C@H]2CCCO2)c(=O)c2ccccc12)NCc1ccccc1', 'O=C(CSC(=S)[C@@H]1CCCC[NH+]1C1CC1)Nc1cncc(-c2ccccc2)n1', 'O=C(CSC(=O)C1CCCC1)NC[C@@H](c1ccc(Br)cc1)N(CC1CC1)S(=O)(=O)c1ccccc1', 'O=C(c1cc([N+](=O)[O-])on1)N1CCC[C@@H]1c1ccccc1', 'O=C(c1cc([N+](=O)[O-])ccc1[N+](=O)[O-])N1CCC=CC1', 'O=C(c1cc(Cl)c2[nH]ccc2c1)N1CCN(c2cnccn2)CC1', 'O=C(c1cc(Cl)c2nccnc2n1)N1CCC[C@H]1c1ccc(F)cc1', 'O=C(c1cc([S@](=O)c2cccs2)ccc1N1CC[NH+](CC(=O)N2CCOCC2)CC1)[C@H]1CCOC1', 'O=C([C@H](c1c(Cl)cccc1Cl)N1CCCC1)N(C1CCCCC1)S(C)(=O)[O-]', 'O=C([O-])/C(=N\\O)c1ccccc1Br', 'O=C([O-])/C(=N/c1ccccc1)c1ccc(Br)o1', 'O=C([O-])/C(=C\\C(=O)Nc1ccccc1OC(F)(F)F)Nc1nc(C2CC2)n(-c2ccccc2)n1', 'O=C([O-])/C(=C\\c1ccc2ccccc2c1)C1CCCCC1', 'O=C([O-])c1[nH]nc(-c2cccc([N+](=O)[O-])c2)c1-c1ccccc1C(F)(F)F', 'O=C([O-])c1[nH]ncc1Cn1c(CO)nc2ccc(-c3ccccc3)nc21', 'O=C([O-])/C(=N/c1c(Cl)cccc1[N+](=O)[O-])N1CCCC1', 'O=C([O-])/C(=N/O)N1CCOCC1', 'O=C([O-])/C(=N/Nc1ccccc1Cl)c1ccncc1', 'O=C([O-])/C(=N/n1cccn1)Nc1ccc(Br)cc1C', 'O=C(/C(c1cccs1)(c1ccccc1)c1ccc2c(c1)C=CC2)c1ccccc1', 'O=C(/C(c1cccs1)(Br)F)C(F)(F)F', 'O=C(/C(CO)Cc1ccccc1OC(F)F)CCc1ccsc1', 'O=C(/C(CO)(CC(=O)N1CCOCC1)c1ccoc1)c1ccccc1', 'O=C(/C(CO)c1ccc(Cl)cc1)c1ccc(Br)c([N+](=O)[O-])c1', 'O=C(/C(CO)=O)c1cc(F)ccc1Oc1ccccc1', 'O=C(/C(CO)[C@@H]1CCC[C@@H](C(F)(F)F)C1)c1ccc([N+](=O)[O-])cc1', 'O=C(/C=C1CCCC[C@@H]1C(=O)NCc1coc(-c2cccs2)n1)c1ccccc1Cl', 'O=C(/C=C1CC(=O)c2ccccc2[C@@H]1c1c[nH]c2ccccc12)c1cccc(Cl)c1', 'O=C(/C=C1CC=C(c2ccc(F)c(F)c2)C1)[C@H]1COc2ccccc2C1', 'O=C(/C=C1CCOC[C@@H]1N1C[C@H]2CCCC[C@H]12)N1CCOCC1', 'O=C(/C=C1CCN(C(=O)CCc2ccccc2)N=N1)Nc1ccccc1F', 'O=C(/C=C1CCSc2ccccc21)c1cc(F)ccc1F', 'O=C(COc1cc([N+](=O)[O-])ccc1NC(=O)N1CCN(c2cc[nH+]cc2)CC1)c1ccc(Cl)cc1', 'O=C(COc1cc([N+](=O)[O-])c([O-])cc1Br)NCc1ccc(Cl)cc1', 'O=C(COc1cc(Cl)c2[nH]c3c(c2c1)CCCC3)NCC1CC[NH+](Cc2ccccc2)CC1', 'O=C(COc1cc(Cl)c2ccccc2c1O)NNC(=O)COc1ccccc1', 'O=C(COc1cc(Cl)c2ncn(-c3ccccc3[N+](=O)[O-])c2c1)Nc1cc(C(F)(F)F)ccc1Cl', 'O=C(COc1cc(F)c(Br)cc1F)N1CCC[C@@H]1c1ccccc1', 'O=C(COc1cc(F)ccc1Cl)NN[C@H]1CCOc2ccccc21', 'O=C(COc1cc(C2CCN(c3ncccn3)CC2)on1)NCc1ccccc1', 'O=C(COc1cc(C2(O)CCCCCC2)cc(Br)c1)NCC1CCCCC1', 'O=C(COc1cc(C2[C@H](C(F)(F)F)N=N2)[nH]n1)N1CCN(c2ccc(F)cc2)CC1', 'O=C(Cc1coc2ncccc12)NCCn1nc(-c2ccccc2)ccc1=O', 'O=C(Cc1conn1)NCCN1CCSC[C@@H]1C[NH+](C)C', 'O=C([C@H](C#N)c1cccs1)N1CCC[C@@H](c2ncccn2)C1', 'O=C(CNC(=O)CCc1c[nH]c2ccccc12)c1ccccc1', 'O=C(CNC(=O)C(F)(F)F)N[C@@H](C(=O)c1ccc(Cl)cc1)C1CCCC1', 'O=C(CNC(=O)Cc1ccccn1)NCC1(c2ccc(F)cc2)CCC1', 'O=C(CNC(=O)C1(c2ccccn2)CCOCC1)Nc1cnn(Cc2cc(F)ccc2)c1', 'O=C(CNC(=O)CS[C@@H]1N=NC=C1O)NC[C@@H](CO)c1ccccc1', 'O=C(CNC(=O)C[NH+]1CCN(c2c(F)cccc2F)CC1)Nc1cccc(Cl)c1', 'O=C(CNC(=O)C[NH2+]C1CC1)N1CC[C@](O)(CO)C1', 'O=C(/N=c1/oc(Cc2ccccc2)cc2c1N[C@H](c1ccco1)S2)c1ccc(F)cc1', 'O=C(/N=c1/occ2ccccc2c1Cl)c1ccc2c(c1)OCCO2', 'O=C(C1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1)N1CCC(c2nc(-c3cccnc3)c[nH]c2=O)CC1', 'O=C(CCOn1cccn1)Nc1ccc2c(c1)OCO2', 'O=C(CC[NH+]1CCC[C@H](c2nc(C3CC3)no2)C1)Nc1ccc(Cl)cc1', 'O=C(c1cc(Br)c[nH]c1=O)[C@@H]1CCC[NH+](Cc2ccc(Cl)cc2)C1', 'O=C(c1cc(Br)c[nH]1)N(Cc1ccncc1)Cc1cccnc1', 'O=C([C@H](Nc1ccccc1)C(F)(F)F)Nc1ccccc1C#N', 'O=C([C@H](c1c(F)cccc1Cl)N(Cc1c[nH]c2ccccc12)C(F)(F)F)N1CCCC1', 'O=C([C@H](Nc1nc(-c2ccccc2)n[nH]c1=O)Nc1ccccc1)N1CCCCC1', 'O=C([C@H](Nc1ncccn1)c1ccccc1)[C@H]1CCCO1', 'O=C([C@H](Nc1nc2c(Cl)cccc2s1)c1cccs1)Nc1c(F)cccc1F', 'O=C([C@H](Nc1nc[nH]n1)C1CCCC1)N(C1CCCCC1)[C@@H]1CCS(=O)(=O)C1', 'O=C(C1CC[NH+]([C@H](c2ccccc2)[C@H]2CCCO2)CC1)N1CCOCC1', 'O=C(C1CC[NH+]([C@H]2CCO2)CC1)N1CCN(C(=O)Nc2ccccc2)CC1', 'O=C([C@H](Sc1ccccn1)C1CC1)N1CCN(c2ccccc2F)CC1', 'O=C([C@H](Sc1ccsc1)c1ccc(F)cc1)N1CCc2ccccc21', 'O=C([C@H](Sc1cc(Cl)ccc1Cl)n1cccn1)Nc1n[nH]c(=O)[nH]c1=O', 'O=C([C@H](Sc1ccc(Cl)cc1)C(=O)NCc1ccccc1)NC1CCCC1', 'O=C(c1cccs1)[C@H](CO)c1ccncc1', 'O=C(c1cccs1)[C@H]1CCCN(c2ccccc2)C1', 'O=C(c1cccn1CCO)N1CCC(OC2CCCCCC2)CC1', 'O=C(c1cccn1Cc1ccccc1)N1CCc2[nH]c[nH+]c2C1', 'O=C(c1cccn1C1CCCC1)N1CCOc2ccccc2[C@@H]1c1cc(F)cc(F)c1', 'Oc1cc(C(F)(F)CN2CCOCC2)cc(Cl)c1OCc1ccccc1', 'Oc1cc(C(F)(F)F)ccc1/C=N/SC1=NC2=NC=NC2=N1', 'O=[N+]([O-])c1ccc2oncc2c1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1C[NH+]1CC[C@H](C[NH+]2CCCC2)C1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1O', 'O=[N+]([O-])c1ccc2[nH]ncc2c1N[C@H]1CCCOc2ccc(F)cc21', 'O=[N+]([O-])c1ccc2[nH]ncc2c1-c1csc(-c2ccccc2)n1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1Sc1ccccc1F', 'O=[N+]([O-])c1ccc2[nH]ncc2c1/N=N/c1csc(Br)c1F', 'Oc1cc(O)c([N+](=O)[O-])cc1OC[C@H]1CCCO1', 'Oc1cc(-n2nncc2CN2CCO[C@@H](c3ccccc3)C2)ccc1N1CCCC1', 'Oc1cc(-n2nncc2CN2CCN(Cc3ccccc3)CC2)ccc1[N+](=O)[O-]', 'Oc1cc(-n2nncc2NCCc2ccc(CCl)cc2)ccc1O', 'Oc1cc(-n2nncc2[C@@H]2CSCc3ccccc32)cs1', 'Oc1cc(-n2nncc2SCc2cn(-c3ccccc3)nc2-c2ccc(F)cc2)ncc1Cl', 'Oc1cc(-n2nncc2Br)c2ccccc2c1O', 'Clc1cccc2[nH]nc(-c3nnc(-c4cccc(Cl)c4F)[nH]c13)CC2', 'O=C([C@H](c1c(F)cccc1F)N1CCN(c2ccccc2)CC1)N1CCCC1', 'C=C1c2c(cccc2C)[C@H]1CC(=O)N[C@@H]1CCc2cc(SC)ccc21', 'C=C1c2ccc(Cl)cc2CCN(C(=O)[C@H]2CC=CCC2)C1', 'Oc1cccc2cc(O[C@H](Cc3ccccn3)[NH+]3CCCC3)nnc12', 'Oc1cccc2cc(O[C@H]3CCC[NH2+]C3)ncc12', 'Nc1cnn(-c2ncccn2)c1Br', 'Nc1cnn(C2CCN(c3ccc(C#N)cc3)CC2)n1', 'Nc1cnn([C@@H]2CCC(=O)N(C3CCCCC3)C2)c(=O)n1', 'Nc1cn[nH]c(=S)n1', 'NS(=O)(=O)/C=C(\\Cc1ccccc1)I', 'NS(=O)(=O)/C=C(/Cc1ccc(O)cc1)c1ccccc1', 'NS(=O)(=O)/C=C(c1cccs1)S(=O)(=O)Cc1ccccc1', 'NS(=O)(=O)/C(=C\\NCc1ccccc1)n1cc(Br)c([N+](=O)[O-])c1', 'NS(=O)(=O)/C(=C\\c1ccco1)CN1C(=O)CN(CCO)C1=O', 'NS(=O)(=O)/C=C\\c1ccc(-n2cccc(C(=O)Nc3ccccc3)c2=O)cc1', 'NS(=O)(=O)/C=C/CC(=O)Nc1cnn(C)c1', 'NS(=O)(=O)/C=C/C(=O)c1ccccc1', 'NS(=O)(=O)/C=C/Cc1ccccc1', 'NS(=O)(=O)/C=C/C1CC1', 'NS(=O)(=O)/C=C/C=C/c1ccccc1', 'NS(=O)(=O)/C=C/COc1cccc(F)c1[N+](=O)[O-]', 'NS(=O)(=O)/C=C/CSc1ccc(F)cc1', 'NS(=O)(=O)/N=C(/NCc1ccccc1)NC1CCN(c2ccccc2)CC1', 'NS(=O)(=O)/N=C(/[O-])c1ccccc1C1([NH+]2CCCCC2)CCSCC1', 'NS(=O)(=O)/N=C(/c1ccccc1)[C@@H]1CCCN(c2ccc(Cl)cc2)C1', 'NS(=O)(=O)N1Cc2ccccc2S(=O)(=O)[C@@H]1Cc1ccccc1-n1cccc1', 'NS(=O)(=O)N[C@@H](CCCn1cc[nH+]c1)c1ccc(F)cc1', 'NS(=O)(=O)N[C@@H](c1ccccc1)c1csc2ccccc12', 'NS(=O)(=O)/C(C#N)=C1\\C=CC(=O)N[C@@H]1c1ccccc1', 'Nc1[nH]nc(NCc2ccccc2)c1Cl', 'Nc1[nH]nc(N(CCc2ccccc2)C2CCCCC2)c1[N+](=O)[O-]', 'Nc1[nH]nc(Nc2ccccc2F)c1[N+](=O)[O-]', 'Nc1[nH]nc(N2CCC[C@@H](c3nc4ccccc4[nH]3)C2)c1N', 'Nc1[nH]nc(NNC(=O)[C@@H]2COc3ccc(Br)cc32)n1', 'Nc1[nH]nc(N[C@@H](c2cccs2)c2cccs2)c1Cl', 'Nc1[nH]nc(N/N=c2/[nH]c3ccccc3n2Cc2cccs2)c1C(F)(F)F', 'Nc1c(Cl)c([O-])c2cc(F)ccc2c1S(=O)(=O)/N=C(\\[O-])C12CC3CC(CC(C3)C1)C2', 'Nc1c(Cl)cc(CNC(=O)c2scnc2C2CC2)cn1', 'Nc1c(Cl)c2c(cc1Cl)c1c([nH]c1=O)CCCC2', 'NS(=O)(=O)N[C@H]1c2ccccc2C[C@@H]1OC(=O)/C(C#N)=C/c1ccc(Cl)cc1', 'N=CCn1ncnc1SCC(N)=O', 'N=CC[C@@H](O)C[NH+]1C[C@H](C)C[C@@H](C)C1', 'N=CC[C@@H]1C(=O)N[C@]2(CCc3ccccc32)C1(Cl)Cl', 'O=C(CS(=O)(=O)[N-]c1ccccc1C(=O)c1ccc(F)cc1)NNC(=O)c1cccs1', 'O=C(CSCc1noc(=O)o1)Nc1ccc(F)cc1Cl', 'O=C(CSCc1n[nH]c2ccccc12)n1cccn1', 'O=C(CSCc1ncc(Cl)cn1)NC[C@@H]1CCc2ccccc2O1', 'O=C(Cc1nc2ccccc2[nH]1)NNc1ccccc1', 'O=C(Cc1nc2ncccn2c1C[NH2+]C1CC1)NC1CCCCC1', 'O=C(Cc1cc(Cc2ccccc2)no1)c1cccc(Cl)c1', 'O=C(Cc1cc(=O)n2ccsc2n1)Nc1ccccc1N1CCCC1', 'O=C(Cc1cc(N2CCCC2)ncn1)N1CCCCCC1', 'O=C(Cc1cc(F)ccc1O)NNC[C@@H](O)c1cscc1F', 'O=C(Cc1cc(S(=O)(=O)N2CCOCC2)ccc1O)c1ccccc1', 'O=C(Cc1cc(Cl)ccc1F)Nc1ccccc1-c1ncccn1', 'O=C(CS(=O)(=O)[N-]C(=O)CCCc1cccc(Cl)c1)Nc1ccccc1', 'O=C(CS(=O)(=O)[N-]Cc1cccc(Cl)c1)NCCC(F)F', 'O=C(CS(=O)(=O)[N-]C1CCCCC1)Nc1cccc(C2CCCC2)c1', 'O=C(CS(=O)(=O)[N-]C[C@@H](O)c1ccccc1)Nc1ccc(F)cc1', 'O=C(CS(=O)(=O)[N-]C[C@H]1CCC[NH+]1Cc1cccs1)c1ccc(Cl)cc1', 'O=C(/C(c1cccs1)[NH+](C)Cc1ccc(Cl)cc1)C1(N2CCOCC2)CCCC1', 'O=C(/C(c1cccs1)[NH+]1CCN(C(=O)c2cnn(-c3ccccc3)c2-c2ccccc2)CC1)c1ccc(C(F)(F)F)cc1', 'O=C(/C(c1cccs1)[C@@H](c1ccco1)N1C2=C(CCCC2=O)c2ccccc21)N1CCNC1=O', 'O=C(/C(c1cccs1)[C@@H]1COCCN1c1cccs1)NCc1ccncc1', 'O=C(/C(c1cccs1)c1ccccc1Cl)[C@H]1C[C@H]1c1cccc(Cl)c1', 'O=C(/C(CNc1ccccc1)c1ccccc1)c1ccncc1', 'O=C(/C(CN1CCN(c2ccccc2)CC1)c1ccccc1)S[C@H]1CCS(=O)(=O)C1', 'O=C(/C(CNc1ccccc1Cl)c1ccc([N+](=O)[O-])cc1)c1ccccc1F', 'O=[N+]([O-])c1c(OCc2[nH]c3ccccc3n2)nc2ccccc2c1N1CCOCC1', 'O=[N+]([O-])c1c(OCc2ccc(F)cc2)csc1NCc1ccc(F)cc1', 'O=[N+]([O-])c1c(OCc2nsc(-c3cccs3)n2)oc2ccccc12', 'O=[N+]([O-])c1c(OCOc2cccc(Cl)c2Cl)cccc1[N+](=O)[O-]', 'O=[N+]([O-])c1c(OCO)sc2cc(O[C@H](CSc3ccccn3)[NH+]3CCCCC3)ccc12', 'O=[N+]([O-])c1c(OCO)oc2c1COCCN(c1ccccc1)C2', 'O=[N+]([O-])c1c(OCO)nc(-c2cccnc2)n1C[C@@H]1CCCO1', 'ClCc1ccc2nsnc2c1C[NH+]1CCCC1', 'ClCc1ccc2nc(-c3ccncc3)n[nH+]2c1', 'ClCc1ccc2nnc(-c3ccc(Br)c(F)c3)n2n1', 'Oc1ccc(O[C@H]2C=[NH+]C3=C(C(=O)c4ccc(Cl)cc43)[C@H]2/C=C/OCc2ccccc2)cc1OCC(F)(F)F', 'Oc1ccc(O[C@H]2C=[NH+]C\\C=C2c2cc(Cl)ccc2O)cc1', 'Oc1ccc(O[C@H]2C=[NH+]N=C3CCC[C@H]32)cc1C[NH2+][C@H]1CCS(=O)(=O)C1', 'Oc1ccc(O[C@H]2C=[NH+][C@@H](COc3cc(C(F)(F)F)ccc3Cl)C2)cc1', 'Oc1ccc(O[C@H]2C=[NH+]/C(=C/c3cccnc3)SC2=S)cc1', 'C[C@@H]([C@@H](O)c1ccc2c(c1)CCCC2)[NH+](C)Cc1ccccc1O', 'O=C(CC1(C2=Nc3ncnn23)[C@@H](CO)C1)N1CCCCC1', 'O=C(CC1(O)CCOCC1)N1CCN(Cc2cccc(C(F)(F)F)c2)CC1', 'O=C(CC1([C@@H](O)c2ccccc2)CC1)N1CCC2(CC1)OCCO2', 'O=C(CC1([O-])CCC1)NCCc1nc2ccccc2o1', 'O=C(CC1([NH3+])CCC1)N1CCCCC1', 'O=C(c1cn2ccsc2c1NC[C@H](O)c1ccc(Br)cc1)N1CCC(O)CC1', 'O=C(c1cn2ccsc2c1N)c1cc(Br)ccc1OCc1ccccc1', 'O=C(c1cn2ccsc2c1Nc1ccc(Br)cc1F)N1CCO[C@@H](c2ccccc2)C1', 'O=C(c1cn2ccsc2c1N1C[C@@H]2CCCC[C@H]2C1)Nc1cccc2cccnc12', 'O=C(c1cn2ccsc2c1N[C@@H]1CCOC1)N1CCC[C@@H](c2nnc3n2CCCCC3)C1', 'O=C([O-])[C@H]1COc2cc(Cl)ccc21', 'O=C([O-])[C@H]1OC(c2ccc([N+](=O)[O-])cc2)C(=O)N=C1N1CCCCC1', 'O=C([O-])[C@H]1[C@H]2C=C[C@@H]1[C@@H](c1ccc(OC(F)F)cc1)N2', 'O=C([O-])[C@H]1[C@@H]2c3cccc(F)c3CC[C@H]1O2', 'COc1nn(C)cc1C(=O)Nc1nc(-c2ccccc2)c(C)s1', 'COc1nn(-c2ccccc2)c(C(=O)N[C@H]2C[C@]2(c2ccc(Br)cc2)c2ccco2)c1C', 'COc1nn(C)c(CC)c1CNCCNC(=O)c1ncccc1OC(C)C', 'COc1nn(C)cc1CN(C)C(=O)N[C@@H]1CCN(c2cnn(C)c2)C1', 'COc1nn(-c2cccc(C(=O)Nc3cc(Cl)c(OC)cc3)c2)c(C)c1Br', 'COc1nn(-c2ccccc2)c(C(C)C)c1NS(=O)(=O)c1cccc(C(F)(F)F)c1', 'COc1nn(-c2c(C)cccc2F)c2cccnc12', 'COc1nn(-c2cccc(C(N)=O)c2)c(C(=O)[O-])c1C', 'COc1nn(-c2c(C)cc(C)c(C)c2NC(=O)C(=O)N[C@@H]2CCCC[C@@H]2C)cn1', 'COc1nn(-c2c(NC(=O)c3ccccc3)c3ccccc23)cnn1C', 'COc1nn(-c2c(F)cccc2Cl)c2c1[C@@H](c1ccc(OC)cc1C)C2', 'COc1nn(-c2c(Cl)cccc2Cl)c(-n2nc(C)cc2C)c1NC(=O)c1ccc(NC(C)=O)cc1', 'COc1ncnc(NCc2cccc(F)c2)c1Br', 'COc1ncnc(N(C)[C@@H]2CCO[C@]3(CCSC3)C2)c1C', 'COc1ncnc(N)c1OC[C@@H]1C[C@]12CCc1ccccc12', 'COc1ncnc(Nc2ccc(Cl)cc2[N+](=O)[O-])c1N', 'COc1ncnc(N2CCN(c3ccccc3OCc3ccccc3)CC2)c1[N+](=O)[O-]', 'COc1ncnc(NNC(=O)c2cc3ccccc3nc2-c2ccccc2)n1', 'COc1ncnc(N[C@@H]2CCO[C@H]2c2ccccc2F)c1[N+](=O)[O-]', 'COc1ncnc(N[C@H]2[C@@H]3CC[C@H](C2)[C@H]2C(=O)N[C@H]2C3C(=O)C(C)(C)C)c1O', 'Clc1cc(Br)c2nc(OC3CCCC3)oc2c1', 'Clc1cc(Br)c2nc(-c3ccccc3)[nH]c2c1', 'Clc1cc(Br)c2nc([C@H]3CCC[NH2+]C3)sc2c1', 'Clc1ccc(-c2nonc2C2CC2)cc1', 'Clc1ccc(-c2nonc2[S-])cc1', 'Clc1ccc(-c2nonc2Cn2cncn2)cc1', 'Clc1ccc(-c2nonc2N2CCc3sccc3C2)cc1', 'Clc1ccc(-c2nonc2)cc1', 'Clc1ccc([C@@H](Cl)[C@H]2C[C@@H]2CC(=O)[O-])cc1', 'Clc1ccc(COCc2ccccc2)[nH+]c1', 'Clc1ccc(CO)cc1SCC[C@H]1CCC[NH+](C(=O)Cc2ccccc2)C1', 'Clc1ccc(COc2ccc(Br)cc2)cc1', 'Clc1cc(/C=N/OCc2ccccc2)co1', 'Clc1cc(/C=N/N2CCN(C3CCCCC3)CC2)ccc1Br', 'Clc1cc(/C=N/n2cc(S(=O)(=O)Cc3ccccc3)ccc2=O)ccc1C1CC1', 'Clc1cc(/C=N/Sc2cnn(-c3ccccc3)c2Br)ccc1[N+](=O)[O-]', 'Clc1cc(Br)cc(CN2CCO[C@@H](c3ccc(C(F)(F)F)cc3)C2)c1', 'Clc1cc(Br)cc(Oc2ccccc2Br)c1', 'Clc1cc(Br)cc(I)c1Oc1ccccc1', 'Clc1cc(Br)cc(F)c1C(=O)Nc1nccc(-c2cnccn2)n1', 'Clc1cc(Br)cc([C@@H]2CCCN2C(=O)c2cccnc2)n1', 'Clc1cc(Br)cc(-c2ccncc2)c1', 'Clc1cc(Br)cc(Cl)c1OCC[NH3+]', 'Clc1cc(Br)cc(/C=N/NCC2[NH+]c3ccccc3C2=O)c1', 'Clc1cc(F)cc(Oc2ccncc2F)c1', 'Clc1cc(F)cc(I)c1O', 'Clc1cc(F)cc(NCc2cccc(Cl)c2)c1', 'Clc1cc(F)cc(F)c1N1CC[NH+](C[C@H](O)CN2Cc3ccccc3C2)CC1', 'Clc1cc(F)cc(-c2ccncc2)c1C[NH2+][C@H]1CC[NH+](Cc2ccccc2)C1', 'Clc1cc(F)cc(SCc2nc(Cc3ccccc3)cs2)c1', 'Clc1cc(F)cc(Cl)c1C(F)(F)F', 'Clc1cc(F)cc([C@H]2CCCN2CCn2cnnc2)c1', 'Clc1ccc(-c2csc3s/Cc4ccccc4c23)cc1', 'Clc1ccc(-c2csc3scc(-c4ccc(Cl)c(F)c4)c3c2=O)cc1', 'Clc1ccc(-c2csc3[nH]c([C@@H]4CCCNC4=O)nc23)cc1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1', 'Clc1ccc([C@@H](NCc2nccn2C2CCCCC2)c2ccccc2)cc1', 'Clc1ccc([C@@H](Nc2ccc(Br)cc2)C(F)(F)F)cc1Cl', 'Clc1ccc([C@@H](NNC(=O)c2ccccc2Cl)N2CCCCC2)cc1Cl', 'Clc1ccc([C@@H](N[C@@H]2CCc3ccccc32)c2ccccc2)cc1', 'Clc1ccc([C@@H](NS(=O)(=O)c2ccc(F)cc2)NC(=O)C2CC2)cc1', 'Clc1ccc(C(=O)N2CCN(Cc3ccc(Cl)cc3)CC2)cc1', 'Clc1ccc(C(CCBr)OCc2ccccc2)cc1', 'Clc1ccc(C(N)=O)c([N+](=O)[O-])c1Sc1ccccc1', 'Clc1ccc(C(F)F)cc1NC[C@H]1CCCN(c2ccccc2)C1', 'Clc1ccc(CCC[NH2+][C@@H]2CCCc3cc(OCc4ccc(Cl)cc4)sc32)cc1', 'Clc1ccc(CC(=O)[C@@H]2CSCN2C(F)(F)F)cc1', 'Clc1ccc(CC2=NC(=O)/C3=NC4=CC(NC(=O)CS/C4=NC/C(=C/c4ccccn4)C3)C2)cc1OCCO', 'Clc1ccc(CCOC(F)F)c(Br)c1', 'Clc1ccc(CCNC[C@@H]2CCOc3ccc(F)cc32)cc1', 'Clc1ccc(CC[C@@H](CBr)C2CC2)cc1', 'Clc1ccc(CC[C@H]2CCCN(c3ncccn3)C2)cc1', 'Clc1ccc(CC[NH+]2CCC(c3nccs3)CC2)cc1', 'Clc1ccc(CCBr)c(CN2CC[NH2+]CC2)c1', 'Clc1ccc(CC[NH2+][C@H]2CCC[C@@H](O)C2)cc1', 'Clc1cc(/C=C/[N+](=O)[O-])cc(C(=O)[O-])c1', 'Clc1cc(/C=C/C[NH+](Cc2ccco2)Cc2ccco2)n(C2CC2)c1', 'Clc1cc(/C=C/N=C(\\Nc2ccccc2)c2ccccc2)ccc1Cl', 'Clc1cc(COC2CC2)on1', 'Clc1cc(CO[C@H]2CSCCS2)c(O)cc1O', 'Clc1cc(Nc2n[nH]nc2C2CC2)ccc1Cl', 'Clc1cc(Nc2nc3ccccc3o2)ccc1OCc1ccccc1Cl', 'Clc1cc(Nc2c(Cl)cc(Cl)c(Cl)c2Cl)ccc1F', 'Clc1cc(Nc2ccccc2Cl)nc(-c2ccccn2)n1', 'Clc1cc(Nc2nc(C3CC3)no2)c2ncccc2c1F', 'Clc1cc(Nc2ncc[nH]2)nc2ccccc12', 'Clc1cc(Nc2nc3cc(Cl)ccn3n2)c2c(nc3ccccc13)CCC2', 'Clc1cc(Nc2ncnc3c2CCCCC3)ccc1Cl', 'Clc1cc(Nc2nc[nH]n2)ccc1O', 'Clc1cccc([C@@H](O)CNc2cccc(F)c2)c1', 'Clc1cccc([C@@H]2CC[C@@H]2CN2CCO[C@@H](c3ccccc3)C2)c1', 'Clc1cccc(SC[C@@H]2CCC[NH2+]2)c1', 'Clc1cccc(S(=O)(=O)/N=C(\\[O-])[C@@H]2CCOC3(CCCC3)C2)c1', 'Clc1cccc(S[C@@H]2CCN(c3ccccn3)C2)c1', 'Clc1cccc(S[C@H]2C[NH2+]CCN2C(=O)c2ccccc2)c1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n', 'Clc1ccsc1-c1cscn1', 'Clc1ccsc1-c1c[nH]c(=O)[nH]c1=O', 'Clc1ccsc1-c1cc(COc2cccc(Cl)c2Cl)on1', 'Clc1ccsc1-c1cncc(Br)c1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1/N=C/c1ccc(Sc2ncc[nH]2)o1', 'Clc1ccsc1/N=C(\\[O-])COc1ccc(C(F)(F)F)cc1', 'Clc1ccsc1/N=C\\C(=O)NCCN1C(=O)/C(=C/c2ccc(Cl)cc2)SC1=S', 'Clc1ccsc1/N=C1/S[C@H](CC(=O)NC2=NN=CC3)CCCC3=C2C=C1', 'Clc1cc(/N=C/c2cccs2)ccc1Cl', 'Clc1cc(/N=C/c2sccc2C(F)(F)F)ccc1Cl', 'Clc1cc(/N=C/c2ccc(I)cc2)nc(Oc2ccc(O)cc2)c1', 'Clc1cc(/N=C/c2nc3ccccc3o2)ccc1Oc1ccc([N+](=O)[O-])cc1[N+](=O)[O-]', 'Clc1cc(Cl)cnc1C1CCCCC1', 'Clc1cc(Cl)cn2cc(-c3ccccc3)cc12', 'Clc1cc(Cl)c2sc(-c3ccc(CCN4CCOCC4)o3)nc2c1', 'Clc1cc(Cl)c2c(c1)OCO2', 'Clc1cc(Cl)cn2c(-c3ccco3)cccc2n1', 'Clc1cc(Cl)cn2ncnc12', 'Clc1cc(Br)c2nc(-c3ccccc3Cl)oc2c1', 'Clc1cc(Br)c2nc(OCc3cscn3)[nH]c(=O)c2c1', 'Clc1cc(Br)c2nc(O)ncc2c1O', 'Clc1cc(Br)c2nc(Oc3ccccc3F)[nH]c(=O)c2c1', 'Clc1cc(/C=Cc2ccc(COC3CCCC3)cc2)no1', 'Clc1cc(CCl)cc(OCc2ccc(Cl)cc2)n1', 'Clc1cc(C(=O)NCCCc2ccsc2)cs1', 'Clc1cc(C(CO)OCc2ccncc2)n[nH]1', 'Clc1cc(C(F)(F)F)ncc1N1CCCSCC1', 'Clc1ccc(-c2nnnn2CCC(=O)N2CCCC2)cc1', 'Clc1ccc(-c2n[nH]c(Nc3ccc(Cl)cc3)n2)cc1', 'Clc1ccc(-c2n[nH]c3c2C(=O)C[C@@H](c2ccccc2)C3)cc1', 'Clc1ccc(-c2n[nH]cc2C[NH+]2CCCN(Cc3ccc(F)cc3F)CC2)cc1', 'Clc1cc(/C=N\\c2ccc(F)cc2F)ccc1[N+](=O)[O-]', 'Clc1cc(/C=N\\Oc2ccccc2)sc1N1CCCC1', 'Clc1cc(/C=N\\N2CCC[C@@H](C(=O)Nc3ccc([N+](=O)[O-])cc3)C2)ncn1', 'Clc1cc(Br)cnc1[S-]', 'Clc1cc(Br)cnc1Oc1ccccc1F', 'Clc1cc(Br)cnc1I', 'Clc1cc(Br)cnc1N1CCO[C@@H](NC2CC2)C1', 'Clc1cc(Br)cnc1-c1ccc(Cl)cc1', 'Clc1cc(Br)cnc1SCc1ncn[nH]1', 'Clc1cc(Br)cnc1[C@H]1CCCN1S(=O)(=O)c1cccs1', 'Clc1ccccc1-c1csc(CN[C@@H]2CCC[C@@H](C(F)(F)F)C2)n1', 'Clc1ccccc1-c1cncc([C@H]2CCOC2)n1', 'Clc1cccc(NCc2ccc[n+]([O-])c2)c1', 'Clc1cccc(Nc2ncnc(Nc3ccccc3)c2[N+](=O)[O-])c1', 'Clc1cccc(N2CCC/C(=C/c3cccc(C(F)(F)F)c3)SC2=O)c1', 'Clc1cc(/N=C/CC[NH+]2CCC[C@@H]2c2ccc(C(F)(F)F)cc2)ccc1NCC1CC1', 'Clc1cc(/N=C/C(=N/O)c2sccc2C(=O)[O-])ccc1O', 'Clc1cc(/N=C/Cc2ccccc2Cl)ccc1O', 'Clc1cc(/N=C/C2CCOCC2)ccc1OCCCO', 'Clc1cc(/N=C/C=C/c2ccccc2)nc([O-])n1', 'Clc1ccc([C@@H]([O-])CCCNc2ccccc2F)o1', 'Clc1ccc([C@@H](Br)C[C@@H]2CCCO2)cc1', 'Clc1ccc([C@@H](Br)c2ccc(Cl)cc2)cc1', 'Clc1ccc([C@@H](Br)[C@H]2CSCCO2)cc1', 'Clc1ccc([C@@H](Br)[C@@]2(O)CCSC2)cc1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1-c1noc(C(=O)Oc2ccc(F)cc2)n1', 'Clc1ccsc1-c1n[n-]c(C(=O)N2CCCC[C@H]2c2ncccn2)n1', 'Clc1ccsc1-c1nc(Cl)ncn1', 'Clc1ccsc1-c1nnc(N2CCC(O)CC2)o1', 'Clc1ccccc1NS(=O)(=O)c1ccc(C(=O)[O-])cc1', 'Clc1ccccc1NS[C@H]1CCOC2(CCCC2)C1', 'Clc1cc(F)ccc1-c1cccc(S(=O)(=O)/N=C(\\[O-])CCC[C@@H]2CCCO2)c1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'Clc1ccc(-c2csc3sc(CCNC(=O)Cc4ccccc4)nc23)cc1', 'Clc1ccc(-c2csc3sccc23)nc1', 'Clc1ccc(-c2csc3scnc23)cc1', 'Clc1ccc(-c2csc3sc4c(c23)CCCC4)cc1', 'Clc1ccc(-c2csc3[nH]nc(-c4ccccc4)c23)cc1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1', 'Clc1ccc(-c2csc3[nH]ccc23)cc1O', 'Clc1ccc(-c2csc3[nH]c(=O)c(-n4cccn4)cc23)cc1', 'Clc1ccc(-c2csc3[nH]c4ccccc4c(=O)c23)cn1', 'Clc1ccc(-c2csc3[nH]cnc23)cc1', 'Clc1ccc(Br)cc1\n', 'Clc1ccc(Br)cc1Cl', 'Clc1ccc(Br)cc1SCCCC1[NH+]=c2ccccc2=[NH+]1', 'Clc1ccc(Br)cc1Sc1ccc(Br)cc1', 'Clc1ccc(Br)cc1S(=O)(=O)N1CCCC1', 'Clc1ccc(Br)cc1S[C@H]1c2ccccc2C1(Cl)Cl', 'Clc1ccc([C@@H]([C@@H]O)N[C@@H]2CCc3ccccc32)cc1Cl', 'Clc1ccc([C@@H]([C@@H](C(=O)[O-])Cn2cccc2)C(F)(F)F)cc1Cl', 'Clc1ccc([C@@H]([C@@H]2C[NH2+]CCO2)N2CCOCC2)cc1', 'Clc1cccc2c(OCCOc3ccccc3Cl)ncnc12', 'Clc1cccc2c(NCc3cccc([N+](=O)[O-])c3)ncnc12', 'Clc1cccc2c(-c3cccc(Br)c3)cnn12', 'Clc1cccc2cc[nH]c12', 'Clc1cccc2cc(F)ccc12', 'Clc1cccc2ccccc12', 'Clc1cccc2nc(C[NH+](Cc3ccco3)[C@@H]3CCS(=O)(=O)C3)ccc12', 'Clc1cccc2nc(N3CCCC3)[nH]c12', 'Clc1cccc2nc([C@@H](CO)NNC(=O)c3cccc(Cl)c3)nc([O-])c12', 'Clc1cccc2nc(-c3ccco3)nc([O-])c12', 'Clc1cccc2nc(SC[C@H]3CSCCO3)nn12', 'Clc1cccc2nc(/C=C/C3CCCCC3)nn12', 'Clc1ccccc1OCCOCCN1C(=O)NC2(CCCCC2)C1=O', 'Clc1ccccc1OCc1ccc(NC[C@@H]2COc3ccccc3O2)cc1', 'Clc1ccccc1OC1CCCC1', 'Clc1ccccc1-c1noc(CN2CCCOCC2)n1', 'Clc1ccccc1-c1n[nH]cc1C[NH2+][C@@H]1CCc2ccccc21', 'Clc1ccccc1-c1nc2ccc(Cl)cc2s1', 'Clc1ccccc1-c1nnnn1Cc1ccc(F)cc1F', 'Clc1cccc2ccc(C[NH2+]C(c3ccccc3)c3cccs3)nc12', 'Clc1cccc2cccnc12', 'Clc1cccc([S@@]Cc2cccc3c2OCO3)c1', 'Clc1cccc([S@@](=O)Cc2ccc(-c3ccco3)s2)c1', 'Clc1cccc([S@@]c2cccc(Br)c2)c1', 'Clc1cccc2c(Oc3cccc(Cl)c3)n[nH]c12', 'Clc1cccc2c1CC[C@@H]2c1ccc(Cl)cc1', 'Clc1cccc2c1Oc1nc3c(O)c(O)ccc3nc1n2Cc1ccco1', 'Clc1cccc2c1N(Cc1cnn(Cc3ccccc3F)c1)CCC2', 'Clc1cccc2c1[C@@H](Cn1ccnc1-c1ccc[nH]c1=O)C2', 'Clc1cccc2c1nc(N1CC[C@@H](Oc3ccccc3)C1)OCCCO2', 'Clc1cccc2c1-c1ccccc1C2', 'Clc1cccc2c1[C@H](c1ccccc1)N2C(=O)Cc1ccccc1C(F)(F)F', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1Sc1cc([N+](=O)[O-])ccc1N1CCCCCC1', 'Clc1ccsc1Sc1nc(CSc2ccncc2)c[nH]1', 'Clc1ccc(-c2nnnn2Cc2cccs2)cc1', 'Clc1ccc(-c2nonc2)cc1Cl', 'Clc1ccc(-c2nonc2)nc1COCc1ccccc1[N+](=O)[O-]', 'Clc1ccc(-c2cc[nH]n2)cc1CCNC(=O)c1ccccc1', 'Clc1ccc(-c2ccccc2)nc1CNC[C@@H](O)c1ccccc1F', 'Clc1ccc(-c2cccnc2)c([C@H]2N=c3ccccc3=[NH+]2)c1', 'Clc1ccc(-c2cccs2)o1', 'Clc1ccc(Br)c(F)c1C(F)(F)F', 'Clc1ccc(Br)cc1-c1ccccc1', 'Clc1ccc(Br)cc1-n1cccn1', 'Clc1ccc(Br)cc1O', 'Clc1ccc(Br)cc1OCc1cc(Cl)c2c(c1)OCO2', 'Clc1ccc(Br)cc1O[C@@H]1CC[C@@H]([NH+]2CCCC2)C1', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccccc1Cl', 'Clc1ccc(Br)cc1O[C@H](CO)C(F)(F)F', 'Clc1ccc(-c2nnnn2CC2CC2)cc1', 'Clc1ccc(-c2nnnn2[C@@H]2C[C@@H]2CO)cc1', 'Clc1ccc(-c2nnnn2-c2ccc(F)cc2F)cc1', 'Clc1ccc(-c2nnnn2Cl)cc1', 'Clc1ccc(-c2nnnn2/C=C/CC[NH+]2CCCCCC2)nc1', 'Clc1cccc(Br)c1Oc1ccc2c(c1)OCO2', 'Clc1cccc(F)c1C[S@@](=O)[C@@H]1CC(=O)N(Cc2ccc([N+](=O)[O-])cc2)C1', 'Clc1cccc([C@@H]2C[C@@H]2C[NH+]2CCC(n3cc[nH+]c3)CC2)c1', 'Clc1cccc([C@@H]2O[C@@H](O)CO[C@@H]2c2cccc(OC(F)F)c2)c1', 'Clc1cccc([C@@H]2Nc3cc(OCc4ccco4)cc(Br)c3[C@@H]2C(=O)[O-])c1', 'Clc1cccc([C@@H]2[C@@H](C(=O)c3cc(Cl)cc(Cl)c3)NC2=O)c1', 'Clc1cccc([C@@H]2[NH2+]CC[C@@H]2[C@@H]2CCCN2C(=O)C2CC2)c1', 'Clc1cc(Cl)c2c(c1)CCN2C1CCC(CN2CCOCC2)CC1', 'Clc1cc(Cl)c2c3c(ccc2n1)C[C@](O)(Cc1ccc(F)cc1)N3', 'Clc1cc(F)ccc1C[NH2+]C1C[C@@H]2CC[C@@H](C1)C2', 'Clc1cc(F)ccc1NCC(=O)N1CCC2(CCSCC2)C1', 'Clc1cc(F)ccc1[C@@H](O)Cc1ccc[nH]1', 'Clc1cc(F)ccc1-n1ncc(C(F)(F)F)n1', 'Clc1cc(F)ccc1[C@H](O)c1ccccc1', 'Clc1cc(F)ccc1/C=N/N[C@@H]1CCC[C@]12CCC[C@H]2[NH3+]', 'Clc1cc(Cl)c(C(=O)COC(=O)CCCc2ccco2)cc1F', 'Clc1cc(Cl)c(OCc2ncnn2Cc2ccccc2)c(C(F)(F)F)c1', 'Clc1cc(Cl)c(N2CCOC[C@H]2C2(CO)CC2)cc1Cl', 'Clc1cc(Cl)c(-n2cc(I)c3ccccc32)nc1', 'Clc1cc(Cl)c(S(=O)(=O)C(F)F)c(Cl)c1Cl', 'Clc1cc(Cl)c(Cl)nc1N[C@@H]1CCOc2ccccc21', 'Clc1cc(Cl)c(Br)c(Cl)c1Cl', 'Clc1cc(Cl)c(/N=C/c2cc(Cl)ccc2Cl)c(Br)c1O', 'Clc1cc(Br)c2nc(-c3ccccc3)[nH]c(=O)c2c1', 'Clc1cc(Br)c(Br)cc1NC[C@@H]1COc2ccccc2O1', 'Clc1ccc(Br)c(OCc2csc(-c3cccn3CCC(F)(F)F)n2)c1', 'Clc1ccc(Br)c(O)c1', 'Clc1ccc(Br)c(Oc2ccc(F)cc2)n1', 'Clc1ccc(Br)c(/C=C/C(=O)N2CCC[C@@H]2[C@@H]2CCCC2)c1', 'Clc1ccc(Br)c(/N=c2\\[nH]c3cnccc3n2-c2ccccc2)c1', 'Clc1ccc(Br)c(/[NH+]=C\\c2ccccc2)c1', 'Clc1ccc(Br)c(SCc2ccccc2)c1', 'Clc1ccc(Br)c(S[C@H]2CCO[C@]3(CCSC3)C2)c1', 'Clc1ccc(Br)c(S(=O)(=O)N2CCN(c3ccc(Cl)c(C(=O)[O-])c3)CC2)c1Cl', 'Clc1ccc(Br)c(Sc2nnnn2-c2ccccc2)c1', 'Clc1ccsc1NCC[NH+]1CCCC1', 'Clc1ccsc1NC(=S)NN=C1CCc2ccccc21', 'Clc1ccsc1NCc1ccc(F)c(S(=O)(=O)N2CCCCC2)c1', 'Clc1ccsc1NC1=NS(=O)(=O)c2ccccc21', 'Clc1cc(C[C@@H]2Cc3ccccc3S2)[nH]c1NCc1cc(F)ccc1F', 'Clc1ccc([C@@H]([NH2+]Cc2n[nH]c3ccccc23)C2CCOCC2)cc1', 'Clc1ccc([C@@H]([NH2+][C@H]2CCCc3ccc(F)cc32)C2CC2)cc1', 'Clc1ccc([C@@H]([NH2+][C@@H](CO)c2ccccc2)C2CC2)nc1', 'Clc1nnnc(-c2ccc(Cl)cc2)c1-c1cccc(C(F)(F)F)c1', 'Clc1nnnc(-n2cnnc2COC(=O)c2cccc(OCc3ccccc3)c2)c1Cl', 'Clc1nnnc(Br)c1I', 'Clc1nnnc(Br)n1', 'Clc1cc(Cl)nnc1CC[NH2+]Cc1cccc(C(F)(F)F)c1', 'Clc1cc(Cl)nnc1Oc1ccc(Br)cc1', 'Clc1cc(Cl)nnc1N1CCCCC1', 'Clc1cc(Cl)nnc1-c1ccccc1Cl', 'Clc1cc(Cl)nnc1S[C@@H](C[NH3+])c1ccccc1', 'Clc1cc(Cl)nnc1Cl', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1-c1cn(CCC(=O)N2CCCCCCC2)c2ccccc12', 'Clc1ccsc1-c1cn2ccccc2n1', 'Clc1ccsc1-c1cnnn1CC[C@@H]1CCCC[NH2+]1', 'Clc1nnns1\n\n\n\n\n\n\n', 'Clc1cc(/C=C(\\CCl)C2CC[NH+](Cc3ccccc3F)CC2)cc2[nH]ccc12', 'Clc1cc(/C=C(/Cl)[C@H]2COc3ccccc32)cc([N+](=O)[O-])c1', 'Clc1cc(C2COC2)ccc1OCc1ccc(Br)o1', 'Clc1cc(C2(C(=O)[O-])CCCCCC2)no1', 'Clc1cc(C2OCCC2)ccc1-n1cnnn1', 'Clc1ccc(Br)cc1CC[NH2+]C1(c2ccc(F)cc2)CCOCC1', 'Clc1ccc(Br)cc1C(F)(F)F', 'Clc1ccc(Br)cc1Cc1ccc(Cl)cc1F', 'Clc1ccc(Br)cc1C1CC[NH+](C[C@@H](O)C2CCOCC2)CC1', 'Clc1ccc(Br)cc1COC(=O)c1cccc(F)c1', 'Clc1ccc(Br)cc1C[C@@H]1CCCN1c1nc(F)ccc1F', 'Clc1ccc(Br)cc1C[C@H]1CCC[NH+](CC2CC2)C1', 'Clc1ccc(Br)cc1C[NH+]1CCC(N2CCOCC2)CC1', 'Clc1ccc(Br)cc1C[NH2+][C@@H]1CCC[C@H]1O', 'Clc1nnn(-c2ncc3ccccc3n2)n1', 'Clc1nnn(-c2ncc3nc[nH]c3c2)c1Cl', 'Clc1nnnc(NNc2ccc(C(F)(F)F)cc2)n1', 'Clc1nnnc(NN2CCOCC2)n1', 'Clc1nnnc(NN=CSc2ccc(Cl)cc2)n1', 'Clc1nnnc(NN[C@H]2C[C@@H]3COC[C@@H]3C[C@@H]2C2CC2)c1', 'Clc1cc(N2CCOCC2)nc(-c2ccco2)n1', 'Clc1cc(N2CCCc3ccccc32)ncn1', 'Clc1cc(N2CCc3ccccc32)cc([N+](=O)[O-])c1', 'Clc1cc(N2CCOCC2)nc(C[NH+]2CCC[C@]3(CCC(=O)N(Cc4ccccc4)C3)C2)c1', 'Clc1cc(N2CCN(Cc3cccc(F)c3)CC2)ccn1', 'Clc1cc(N2CC[C@H](Cc3ccccc3)C2)nc2ccccc12', 'Clc1cc(NSc2ccn[nH]2)ccc1N/N=C/Oc1cccnc1', 'Clc1cc(NSc2nc3ccccc3[nH]c2=O)n(-c2ccccc2)n1', 'Clc1cc(/C=C2\\CCc3ccccc32)ccc1O', 'Clc1cc(/C=C2/S[C@H]3CS(=O)(=O)C[C@@H]3N2c2ccccc2)ccc1[N+](=O)[O-]', 'Clc1cc(Br)c(-n2ccnc2)c(Br)c1O', 'Clc1cc(Br)cc2oc(-c3ccc(OC4CCCCC4)cc3)nc12', 'Clc1cc(Br)cc2[nH]cnc12', 'Clc1cc(Br)cc2cccnc12', 'Clc1cc(Br)cc2ncnc(-c3cccs3)c1n2-c1ccccc1', 'Clc1cc(NSc2nc3ccccc3[nH]2)ncn1', 'Clc1cc(NSc2nnnn2Cc2ccccc2)cs1', 'Clc1cc(Cl)ccc1CSc1ncccn1', 'Clc1cc(Cl)cc2c1NN(c1ccccc1C(F)(F)F)NC2', 'Clc1cc(/C=N/N(c2ccccc2Cl)C2CC2)c[nH]1', 'Clc1cc(/C=N/Nc2nc(-c3ccccc3)n[nH]c2=O)cs1', 'Clc1cc(/C=N/N2CCCCC2)c2c(c1)OCO2', 'Clc1cc(Cl)c2sccc2n1', 'Clc1cc(N2CCNC[C@@H]2c2cccc3ccccc23)ccn1', 'Clc1cc(N2CCN(c3ccc(F)cc3)CC2)ncn1', 'Clc1cc(N2CCCCC2)c([N+](=O)[O-])cc1NCC1CCCC1', 'Clc1cc(N2CCC(C[NH+]3CCCC3)CC2)nc([O-])c1', 'Clc1cc(N2CCC[C@@H](N3CC[NH+](CCO)C3)CC2)c[nH]1', 'Clc1cc(N2CCC[C@H]2c2nnc(-c3ccccc3)o2)ccc1Cl', 'Clc1cc(N2CCC[C@@](O)(C(F)(F)F)C2)ncc1F', 'Clc1ccccc1O/C=C(/C#N)C(=O)N1CCCC1', 'Clc1ccccc1O/N=C(\\[O-])c1c[nH]nc1-c1ccccc1', 'Clc1ccccc1N1N(CCc2ccccc2F)c2ccccc2C1', 'Clc1ccccc1NCCCO/N=C/c1ccccc1[N+](=O)[O-]', 'Clc1ccccc1NC(=O)CSc1nnc(-c2ccccc2)n1C[C@@H]1CCCO1', 'Clc1ccccc1NCc1ccccc1CN1CC[NH+](CC2(CCO)CC2)CC1', 'Clc1ccccc1NC1(c2ncc[nH+]c2Cl)CCOCC1', 'Clc1ccccc1OCc1cccc(C(F)(F)F)c1', 'Clc1ccccc1O/C=C(c1ccccc1)N1CCN(c2ccccc2)CC1', 'Clc1ccccc1O/Cn1cnc(-c2ccc(Cl)c(F)c2)n1', 'Clc1ccccc1Oc1ccccc1F', 'Clc1ccccc1Oc1cc2c(cc1OCCCOCCO)C(=O)CC2', 'Clc1ccccc1OC1CC[NH+](Cc2cccc(Cl)c2)CC1', 'Clc1ccccc1OC1(C(=O)[O-])CCN(Cc2ccc(Cl)cc2)CC1', 'Clc1ccccc1COC(=O)[C@H]1Sc2ccccc2S1(=O)=O', 'Clc1ccccc1CO[C@H]1CCCOC1', 'Clc1ccccc1COc1ccccc1F', 'Clc1ccccc1-c1cn2cc(Br)cnc2c1[O-]', 'Clc1ccccc1-c1cnccn1', 'Clc1ccccc1-c1cnn2c1N=C1CCCC[C@H]1CC2', 'Clc1cccc2[nH]nc(C[C@@H](C(=O)[O-])N3CCCC3c3ccccc3)c12', 'Clc1cccc2[nH]nnc12', 'Clc1cccc2[nH]nnc12', 'Clc1cccc2[nH]nc(-c3ncc(-c4ccccc4)nc3)c12', 'Clc1cccc2[nH]ncc12', 'Clc1cccc2[nH]nnc12', 'Clc1cccc2[nH]nnc1c1cc(C3CC3)nn12', 'Clc1cccc2[nH]nnc12', 'Clc1ccccc1Nc1cccc(C(=O)Oc2ccc(C(F)(F)F)cn2)c1', 'Clc1ccccc1Oc1ccccc1CN(Cc1ccccn1)C1CC1', 'Clc1ccccc1Oc1cnn(C[C@@H]2CCCC[C@@H]2O)c1', 'Clc1ccccc1N1N=C(C(=O)Nc2ccc([N+](=O)[O-])cc2)SC1=S', 'Clc1ccccc1N1NC2=NC=C2OC(=S)N1C1CCCCCC1', 'Clc1ccccc1N1N(C(=O)Cc2ccccc2F)CCC1', 'Clc1ccccc1N1Nc2c(ccc(Br)c2F)C1', 'Clc1ccccc1C1=NN=C[C@@H]1C(=O)NC1(c2ccccc2)CC1', 'Clc1ccccc1C1CC[NH+](Cc2cccc(Cl)c2Cl)CC1', 'Clc1ccccc1C1(C[NH2+]Cc2ccccn2)CC1', 'Clc1cccc2c(N)cc(C[C@@H]3CN4S(=O)(=O)CCC[C@@H]4O3)cc12', 'Clc1cccc2c(N3CCOCC3)nc3ccccc3c12', 'Clc1cccc2c(N[C@@H]3CCOC3Cl)ncnc12', 'Clc1ccc(-c2cc[nH]n2)c(Nc2ccc(F)cc2Br)c1', 'Clc1ccc(C[NH2+]Cc2cc3ccccc3o2)cc1', 'Clc1ccc(C[NH2+]c2cccc(C(=O)[O-])c2)cc1', 'Clc1ccc(C[NH2+][C@H]2CCC[C@@H]2[C@@H]2CCCC2)cc1', 'Clc1ccc(C[NH2+][C@@H]2CCC[C@H](c3nc(C(F)(F)F)cs3)C2)cc1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1S(=O)(=O)Nc1ccc(C(=O)N2CCOCC2)cc1', 'Clc1ccsc1N/C=C1\\NC=CCC1', 'Clc1cc(Cl)cnc1-c1ccccc1', 'Clc1cc(Cl)cnc1C[NH+]1CCc2oncc2C1', 'Clc1cc(Cl)cnc1[N-]c1ccc2ccccc2c1', 'Clc1cc(Cl)cnc1N1CCCCC[C@@H]1C[C@@H]1CCCO1', 'Clc1cc(Cl)cnc1-c1ccc(OC[C@H]2CCCO2)cc1', 'Clc1cc(Cl)cnc1SCC1CCOCC1', 'Clc1cc(Cl)cnc1Cl', 'Clc1nnnn1c1[nH]cnc1C[C@@H]2CCC[NH+](Cc1ccccc1Cl)C2', 'Clc1nnnn1c1[nH]ncc1[C@@H](c1ccccc1)N1C(=O)c2ccccc2C1=O', 'Clc1ccsc1S(=O)(=O)c1ccc(OC2CCCC2)cc1', 'Clc1ccsc1S(=O)(=O)Nc1ccccc1C(=O)c1ccc(Br)cc1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1S(=O)(=O)Nc1ccc(C(F)(F)F)cc1', 'Clc1ccsc1CNC[C@@H]1OCCN(C(=O)c2cccs2)C1', 'Clc1ccsc1CN(Cc1c[nH]c2ccccc12)C1CC1', 'Clc1ccsc1CNc1ccc(F)c(C(F)(F)F)c1', 'Clc1ccsc1CN1CC[C@H]2[C@H](C2)C1', 'Clc1ccsc1CN[C@@H]1CCC(=O)N(C2CCOCC2)C1', 'Clc1ccsc1/C=N/N=C1\\S[C@H]2CS(=O)(=O)C[C@H]2N1CC1CC1', 'Clc1ccsc1/CCNC(=O)NCCCN1CCCCC1=O', 'Clc1ccsc1S(=O)(=O)N1C[C@@H](CO)C1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1S(=O)(=O)N1CC[C@H]2CCCC[C@@H]2C1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1C(F)(F)F', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1Sc1n[nH]c([C@H]2CCC[NH+]2Cc2ccco2)n1', 'Clc1ccsc1Sc1ncccc1C(=O)Nc1ccc(F)c(Cl)c1', 'Clc1ccsc1Sc1nnc(-c2ccccc2)n1C1CCCC1', 'Clc1ccsc1C(F)(F)Cc1ccccc1Cl', 'Clc1ccsc1C[N-]C(=O)c1ccc(Cl)s1', 'Clc1ccsc1C[N-]c1ccc2[nH]ccc2c1', 'Clc1ccsc1C[N-]Oc1ccc2c(c1)CCC2', 'Clc1ccsc1C[N-]F', 'Clc1ccsc1C[N-]Sc1ccc(Br)cc1', 'Clc1ccsc1C[N-][C@H]1[C@H]2CCO[C@@H]2c2ccccc21', 'Clc1ccsc1C[N-]/N=C/C[NH+]1CCCCCC1', 'Clc1nnn(-c2cccc(Cl)c2Cl)n1', 'Clc1nnn(-c2ccccc2)c1-c1ccccc1', 'Clc1nnnn1/N=C\\c1cc([N+](=O)[O-])ccc1O', 'Clc1nnnn1/N=C/c1cc(Cl)ccc1O', 'Clc1nnnn1/N=C/Sc1ccc(C(F)(F)F)cc1Cl', 'Clc1nnnn1/N=C/c1ccc([N+](=O)[O-])cc1', 'Clc1nnnn1/N=C/[C@@H]1CCCC[C@H]1O', 'Clc1ccccc1O/N=C/c1cc(Br)ccc1OCn1cccn1', 'Clc1ccccc1O/N=[N+]=NC(=O)CN1CCN(c2cnccc2)CC1', 'Clc1ccccc1O/N=C(/[O-])N(CC(F)(F)F)C[C@@H]1CCCO1', 'Clc1ccccc1OCCCCNc1ncnc2c1CCC2', 'Clc1ccccc1OCC(=O)N1CCC[C@@H](CCc2ccccc2)C1', 'Clc1ccccc1OCC1CC[NH+]([C@@H]2Oc3ccccc3O2)CC1', 'Clc1ccccc1OCCOc1ccccc1Br', 'Clc1ccccc1OCCNS(=O)(=O)c1ccccc1Br', 'Clc1ccccc1OCCn1cccn1', 'Clc1ccccc1OCCSc1nc(-c2ccco2)n[nH]1', 'Clc1ccccc1OCC[NH+]1CCN(c2nc3ccccc3s2)CC1', 'Clc1ccccc1OCC[NH2+][C@@H]1CCC[C@H](S(C)(=O)=O)C1', 'Clc1ccccc1C(=O)Nc1ccc(Cn2nncc2C2CC2)cc1', 'Clc1ccccc1C(C1CC1)[C@H]1Sc2ccc(Cl)cc2N1c1ccccc1', 'Clc1ccccc1-c1nc(N2CCCC2)nc2ccccc12', 'Clc1ccccc1-c1nccs1', 'Clc1ccccc1-c1csc(Cc2cccnc2)n1', 'Clc1ccccc1-c1nn(-c2ccccc2)c(C(=O)N(Cc2ccccc2)C2CC2)c1C(F)(F)F', 'Clc1ccccc1-c1nnc(SCc2cccc(F)c2)n1C[C@H]1CCCO1', 'Clc1ccccc1-c1nn2c(Cl)cnc2n1Cc1cc(Cl)ccc1Cl', 'Clc1ccccc1Nc1cc(Cl)ccc1OCc1ccccc1F', 'Clc1ccccc1Nc1ncc(Br)cn1', 'Clc1ccccc1CNC[C@H]1CCC[C@H](CNS(=O)(=O)C2CC2)C1', 'Clc1ccccc1CN(Cc1ccsc1)Cc1ccc(F)cc1', 'Clc1ccccc1CNc1nc2ccc(Cl)cc2[nH]1', 'Clc1ccccc1CN1CCC[C@@H](c2ccc[nH+]c2)C1', 'Clc1ccccc1Nc1c2ccc(Oc3ccccc3)nn2c(=O)cn1', 'Clc1ccccc1Nc1ccc2nnc(-c3cccs3)n2n1', 'Clc1ccccc1Nc1cncc(-c2ccc(F)cc2)n1', 'Clc1ccccc1NS(=O)(=O)c1ccc(F)cc1', 'Clc1ccccc1NS(=O)(=O)c1ccc(Br)c([N+](=O)[O-])c1', 'Clc1ccccc1-c1csc(N2CCOCC2)n1', 'Clc1ccccc1-c1cscc1C(F)(F)F', 'Clc1ccccc1NS[C@H](c1ccccc1)N1CCOCC1', 'Clc1ccccc1NS[C@H]1CCCC[C@@H]1C(=O)[O-]', 'Clc1cccc(Br)c1C(F)(F)C[NH+]1CCCC1', 'Clc1cccc2nn[nH]c12', 'Clc1cccc2nn(CCCCNc3cccc(Cl)c3)nc12', 'Clc1cccc2nn(SC[C@@H]3CCCO3)nc12', 'Clc1ccccc1CSCCOc1ccccc1F', 'Clc1ccccc1CS(=O)(=O)CC1CC[NH2+]CC1', 'Clc1ccccc1CSc1nnc(SCc2cccc3ccccc23)n1C1CCCC1', 'Clc1ccccc1-c1csc(C[NH2+]CCO)n1', 'Clc1ccccc1-c1csc(NC[C@@H]2CN(Cc3cccnc3)C[NH+]2Cc2ccccc2)c1', 'Clc1ccccc1-c1csc([C@@H]2C(=O)NC(=O)c3ccc(Cl)cc32)n1', 'Clc1ccccc1-c1csc(-c2cccc(Br)c2)n1', 'Clc1ccccc1-c1csc(Sc2cn[nH]c2-c2cc(Cl)ccc2F)n1', 'Clc1ccccc1-c1csc(Cl)n1', 'Clc1ccccc1-c1csc([C@H]2CCCC[C@@H]2C(=O)[O-])n1', 'Clc1ccccc1-c1csc(/C=C/c2ccncc2)n1', 'Clc1cccc2cc(CNCc3ccccc3F)[nH]c(=O)c12', 'Clc1cccc2cc(OCn3c(CNC(=O)c4cccc(Cl)c4)no3)cnc12', 'Clc1cccc2cc(F)ccc12', 'Clc1cccc2cc(-n3ccnc3)n(-c3ccc(Cl)cc3)c12', 'Clc1cccc2cc([C@H](O)C(F)(F)F)cc(-c3ccccc3)c12', 'Clc1cccc2cc(Br)ccc12', 'Clc1cccc2cc(/C=N/c3cc([N+](=O)[O-])ccc3Cl)[nH]c12', 'Clc1cccc2nn[nH]c12', 'Clc1cccc2[nH]nc(NC[C@@H](O)c3ccccc3)c12', 'Clc1cccc2[nH]nc(S/C=C/C[NH3+])c12', 'Clc1cccc(-c2ccc(C[NH+]3CCCC3)o2)c1', 'Clc1ccccc1Oc1cc(-n2cccn2)nc2cccc(F)c12', 'Clc1ccccc1Oc1cncc2ccccc12', 'Clc1ccccc1Oc1cc([N+](=O)[O-])ccc1O', 'Clc1ccccc1Oc1cc(C(F)(F)F)nc(Nc2ccc(F)cc2)n1', 'Clc1ccccc1Oc1cc(OCc2ccccc2)ncn1', 'Clc1ccccc1Oc1cc(NC(=O)CC2=CCCCC2)ccc1F', 'Clc1ccccc1Oc1cc(F)ccc1O', 'Clc1ccccc1Oc1cc(-c2ccncc2)no1', 'Clc1ccccc1Oc1cc(S(=O)(=O)N2CCCCC2)ccc1Cl', 'Clc1ccccc1Oc1cc(Cl)ccc1F', 'Clc1ccccc1Oc1cc(Br)ccc1C(=O)[O-]', 'Clc1ccccc1Nc1ncnc2[nH]c(C3CC3)nc12', 'Clc1ccccc1Nc1nnc(-c2cc(Br)ccc2O)[nH]1', 'Clc1ccccc1CI', 'Clc1ccccc1CCCC(=O)Nc1ccccc1OCc1ccc(F)cc1', 'Clc1ccccc1CC(C[C@@H](O)CO)NNC(=O)c1ccsc1', 'Clc1ccccc1CCc1cccc(C(=O)N2CCC[C@@H]2c2cccc(F)c2)c1', 'Clc1ccccc1CC1(C[NH2+][C@@H]2CCCc3ccccc32)CCCCCC1', 'Clc1ccccc1CCOc1ccccc1C[NH+]1CCC[C@@H](O)C1', 'Clc1ccccc1CCNCc1cc(=O)n2ncccc2n1', 'Clc1ccccc1CCn1cc[nH+]c(C[NH2+][C@H](CO)c2ccccc2C(F)(F)F)c1=O', 'Clc1ccccc1CC[C@H]1CN(C[C@@H]2CCCCO2)CCO1', 'Clc1ccccc1CC[NH+]1CCCN(Cc2cccc(F)c2)CC1', 'Clc1cccc(F)c1Nc1cccc2ccccc12', 'Clc1cccc(Br)c1CO/N=C1\\C(=O)Nc2ccccc21', 'Clc1cccc(Br)c1OC[C@H](O)COc1ccc2nc[nH]c2c1', 'Clc1cccc(Br)c1N1C[C@@H]2[C@@H](C1)CN(Cc1ccc(Br)cc1)N2', 'Clc1cccc(Br)c1Sc1cccc(Br)c1', 'Clc1cccc(Br)c1Cl', 'Clc1cccc(Br)c1Br', 'Clc1cccc(F)c1C[NH+]1CC[C@H](Nc2ccccc2)C1', 'Clc1cccc(F)c1Oc1ccc(Br)cc1', 'Clc1cccc(F)c1NCc1ccc(F)cc1', 'Clc1cccc(F)c1[C@@H]1CCCN1Cc1ccccc1F', 'Clc1cccc(F)c1-c1cccc(NCc2cccc(F)c2)c1', 'Clc1cccc(F)c1Br', 'Clc1cccc([S@@]c2ccc([O-])nn2)c1', 'Clc1cccc(/C=N/NC(=S)NCc2ccccc2)c1', 'Clc1cccc(/C(CC(=O)c2ccc(OCc3cnn(C)c3)cc2)N2CCCC2)c1', 'Clc1cccc(-c2nnc(SCCC[NH+]3CCCCCC3)nc2-c2ccccc2)c1', 'Clc1cccc(Nc2nccs2)c1Cl', 'Clc1cccc([S@@]CC[NH+]2CCCC2)c1', 'Clc1cccc([S@@]C(C2CC2)C2CC2)c1', 'Clc1cccc([S@@]Cc2cccc3ccccc23)c1', 'Clc1cccc([S@@]C2=NCCc3ccccc32)c1', 'Clc1cccc(/C=C/C=C/CC[NH2+]C2CC2)c1', 'Clc1cccc(/C=N/c2ccc[n+]c2)c1', 'Clc1ccc(-c2nonc2CC2CCCC2)cc1', 'Clc1ccc(-c2nonc2C(F)F)cn1', 'Clc1ccc(-c2nonc2Cc2ccc(Cl)cc2)cc1', 'Clc1ccc(-c2nonc2C2CC2)cc1F', 'Clc1ccc(-c2nonc2COC2CC2)cc1', 'Clc1ccc(-c2nonc2CN2CCN(c3ccccc3)CC2)cc1', 'Clc1ccc(-c2nonc2C[C@@H]2CCCC[NH2+]2)cc1', 'Clc1ccc(-c2nonc2Cn2cnc(-c3ccc(Cl)cc3)n2)cc1', 'Clc1ccc(-c2nonc2C[NH+]2CCCC2)cc1', 'Clc1ccc(-c2nonc2C/[NH+]=C2\\SC3=C(CCCC3=O)NC2=O)cc1', 'Clc1ccc([C@@H]([O-])CNc2ccc3ccc4ccn4c(=O)n(Cc4cccc(F)c4)n3c2)cc1', 'Clc1ccc([C@@H]([O-])c2ccc(OCC[NH+]3CCC(CO)CC3)cc2)cc1', 'Clc1ccc([C@@H]([O-])NC2CCCC2)cc1', 'Clc1ccc([C@@H]([O-])[C@@H](c2ccccc2)N2CC[NH+](CC(=O)NC3CCCC3)CC2)cc1', 'Clc1ccc([C@@H]([O-])[C@H]([NH2+]CC(=O)NC2CC2)C2CC2)cc1', 'Clc1ccccc1NCCCSc1ncnn1C1CC1', 'Clc1ccccc1NCC(=O)N1CCCCC1', 'Clc1ccccc1NCCc1ccc2c(c1)CCCN2Cc1ccccc1', 'Clc1ccccc1NCC1CCN(S(=O)(=O)c2ccc(Br)o2)CC1', 'Clc1ccccc1NCCN1CCC[NH+](C2CCCC2)CC1', 'Clc1ccccc1NCCn1nnc2ccccc21', 'Clc1ccccc1NCCS(=O)(=O)c1ccccc1', 'Clc1ccccc1NCC[C@H]1CCCN1S(=O)(=O)c1c[nH]c2ccccc12', 'Clc1ccccc1NCC[NH+]1CCC(COS(=O)(=O)c2ccccc2)CC1', 'Clc1ccccc1NCC[NH2+]Cc1ccccc1', 'Clc1cccc(NCCNC(=O)c2cccc(F)c2F)c1', 'Clc1cccc(NC(=S)N2CC[NH+](Cc3ccccc3)CC2)c1', 'Clc1cccc(NC2CCCC2)c1', 'Clc1cccc(NC[C@@H]2CC[NH+](CN3CCC(O)CC3)C2)c1', 'Clc1cccc(NC[C@]2(O)CCNC2=O)c1', 'Clc1cc(/C=Cc2[nH+]cn(-c3ccccc3)n2)ccc1NN1CCCC1', 'Clc1cc(/C=Cc2cccc[n+]2[O-])cn1C1CC1', 'Clc1cc(/C=Cc2n[nH]c(-c3ccc([N+](=O)[O-])cc3)n2)ccc1N1CCOCC1', 'Clc1cc(/C=Cc2sncn2)no1', 'Clc1cc(/C=Cc2[nH]nc3c2CCCC3)c(OCC(F)(F)F)cc1', 'Clc1ccccc1O/C=C1/SC(=S)N(CC(=O)Nc2ccccc2)C1', 'Clc1ccccc1O/Cn1nccc1COc1ccc(Cl)cc1Cl', 'Clc1ccccc1O/Cn1[nH+]c(-c2ccccc2)nc1-c1ccccc1Cl', 'Clc1ccccc1O/Cn1cc([N+](=O)[O-])cn1', 'Clc1ccccc1O/Cn1nnnc1Cc1cccs1', 'Clc1ccccc1[C@H]1CCC[NH+](Cc2cccc(O)c2)C1', 'Clc1ccccc1[C@H]1Oc2ccc(Cl)cc2CCN1c1ccc([N+](=O)[O-])cc1', 'Clc1ccccc1[C@H]1NC(=O)NC(=S)N1', 'Clc1ccccc1[C@H]1[C@@H](C(=O)c2ccccc2)O[C@H]1c1ccccc1', 'Clc1ccccc1[C@H]1[C@H](CO)Cc2ccccc2C1', 'Clc1ccccc1[C@H]1[NH+]=c2ccccc2=[NH+]C1=O', 'Clc1cc(F)c(OCC(F)F)cc1I', 'Clc1cc(F)c(O)c(F)c1F', 'Clc1cc(F)c(Oc2cncc(C(F)(F)F)c2)cc1', 'Clc1cc(F)c(O[C@@H]2CCC[C@@H](C[NH3+])C2)cc1', 'Clc1cc(Br)c(SCC(F)(F)F)cc1O', 'Clc1cc(Br)c(S(=O)(=O)[C@@H]2CC[NH2+]C2)cc1F', 'Clc1cc(Br)c(Sc2cccc(F)c2)nc1', 'Clc1ccc(-c2cc[nH]n2)cc1', 'Clc1ccc(-c2cc[nH]n2)o1', 'Clc1ccc(-c2cc[nH]n2)cc1', 'Clc1ccc(-c2cc[nH]n2)nc1', 'Clc1cc(CCl)ccc1Br', 'Clc1cc(CCl)n[nH]1', 'Clc1cc(CNC[C@H](c2ccccc2)c2ccccc2)c(F)cn1', 'Clc1cc(CN(Cc2ccccc2)Cc2ccsc2)ccc1Cl', 'Clc1cc(CNc2ccc(N3CCOCC3)cc2)nc(-c2ccncc2)n1', 'Clc1cc(CN2CCN(Cc3ccccc3)CC2)cc(OCC(F)(F)F)c1', 'Clc1cc(C[NH+](Cc2ccco2)Cc2ccccc2)ccc1Cl', 'Clc1cc(C[NH2+]C2CC2)ccc1N1CCOCC1', 'Clc1cc(C[NH2+][C@H]2CCc3c(Br)cccc32)ccc1Cl', 'Clc1cc(C[NH2+][C@@H]2CSc3ccccc32)ccc1Br', 'Clc1cccc(Br)c1Cl', 'Clc1cccc([S@@](=O)Cc2c[nH]nc2-c2ccccc2)c1', 'Clc1cccc(F)c1NCc1cccc(F)c1', 'Clc1cccc(F)c1N(C[C@@H]1Oc2ccccc2OC1=O)c1ccccc1', 'Clc1cccc(F)c1Nc1ccc([N+](=O)[O-])cc1[N+](=O)[O-]', 'Clc1cccc(F)c1N1CCC(N/C=C2\\C3=C(CCO3)CC2)C1', 'Clc1cccc(F)c1N[C@@H]1CCSc2ccccc21', 'Clc1cccc(F)c1N[C@H]1CC(Cc2ccccc2F)=[NH+]1', 'Clc1cccc(F)c1N/C=C/c1ccc([N+](=O)[O-])cc1', 'Clc1ccc(-c2ccccc2)cc1C[NH2+]C[C@H]1CCS(=O)(=O)C1', 'Clc1cc(F)c(F)cc1C[C@H]1CCO[C@@]2(CCOC2)C1', 'Clc1cccc2[nH]ncc12', 'Clc1cccc2[nH]ncc1n1c(C(=O)OCC(F)(F)F)c1C[NH+](Cc1ccccc1)C2', 'Clc1cccc2[nH]ncc12', 'Clc1ccc(C[NH+](CCCn2cccc2)C2CC2)s1', 'Clc1ccc(C[NH+]2CCC[C@H](c3nc(-c4cccnc4)no3)C2)s1', 'Clc1cc(Br)c2nc([C@H](c3ccc(F)cc3)[NH2+][C@H](c3ccco3)N3CCCCCC3)sc2c1', 'Clc1cc(Br)c2nc([C@H]3[C@@H]4CCCC[C@H]4C3)c[nH]c12', 'Clc1ccccc1O/N=[N+]=SP1CC1', 'Clc1ccccc1O/N=[N+]/C=CSc1ccc2c(c1)OCO2', 'Clc1ccccc1O/N=[N+](\\[O-])c1cccc(S(=O)(=O)N2CCCC[C@H]2C(F)(F)F)c1', 'Clc1ccccc1O/N=[N+]1/N=C/SCCS1', 'Clc1cc(/N=C\\c2ccccc2)cc2c1CCCC2', 'Clc1cc(/N=C\\c2sccc2CO)sc1NCc1ccc(Cl)cc1', 'Clc1cc(/N=C\\c2ccccc2Br)ccc1O', 'Clc1cc(/N=C\\CCn2ccc(Br)c2)ccn1', 'Clc1cc(/N=C\\Cc2ccccc2Br)cs1', 'Clc1cc(/N=C\\C2=CCNC2)c2ccccc2c1Cl', 'Clc1cc(/N=C\\C=C/CC(=O)[O-])c(Br)s1', 'Clc1cc(/N=C\\CSc2ccc(Cl)cc2Cl)ccn1', 'Clc1ccc(Br)c([C@@H](CO)Nc2ccc(Br)cc2)c1', 'Clc1ccc(Br)c([C@@H]2CCC[NH+]2C[C@H]2CO2)c1', 'Clc1ccc(-c2cccs2)nc1', 'Clc1ccc(-c2cc[nH]n2)c(CCC(=O)N2[C@H]3CCn4c(nc5ccccc54)[C@@H]3C2Cc2ccccc2)n1', 'Clc1ccc(-c2cc[nH]n2)c2ccccc12', 'Clc1ccc(-c2cc[nH]n2)cc1Cn1ccccc1=O', 'Clc1ccc(-c2cc[nH]n2)cn1', 'Clc1ccc(-c2n[nH]c(Cc3ccccc3)n2)cc1', 'Clc1ccc(-c2n[nH]c(=S)n2CN2CCOCC2)cc1', 'Clc1ccc(-c2n[nH]c(OCC(F)F)n2)cc1', 'Clc1ccc(-c2n[nH]c(NC3CCCC3)n2)cc1', 'Clc1ccc(-c2n[nH]c(SCn3ccnn2)n3)cc1', 'Clc1ccc(-c2n[nH]c([O-])n2)cc1', 'Clc1ccc(-c2n[nH]c([C@H](c3c[nH]c4ccccc34)N3CCO[C@@H](C(F)(F)F)C3)c2)cc1', 'Clc1ccc([C@@H](O)c2ccc(Br)cc2)c(F)c1', 'Clc1ccc([C@@H](O)[C@@H]2CC(c3ccc(O)cc3)c3ccccc32)cc1', 'Clc1ccc([C@@H](O)[C@H]([NH2+]Cc2ccccc2)C2CCC2)cc1', 'Clc1ccc([C@@H](O)[C@]2(O)CCc3cccnc32)cc1', 'Clc1ccc([C@@H](O)[C@@]2(CBr)CCOCC2)cn1', 'Clc1cccc(Br)c1Br', 'Clc1cccc(S[C@@H](C(=O)CCC2CC[NH+](C3CCOCC3)CC2)c2ccccc2)c1', 'Clc1cccc(S[C@@H]2CCOc3ccc(Cl)cc32)c1', 'Clc1cc(Cl)nn1CCN1CCOCC1', 'Clc1cc(Cl)nn1Cc1cnc2ccccc2c1[O-]', 'Clc1cc(Cl)nn1C[C@@H](O)COc1ccccc1', 'Clc1cc(Cl)nn1C[NH+]1CCN(c2ccccc2)CC1', 'Clc1ccc(Br)c(Cl)c1', 'Clc1ccc(Br)c(NC2(c3ccccc3)N(Cc3ccccc3)C2)c1', 'Clc1ccc(Br)c(Nc2nnc3c(O)nc(SCC(F)(F)F)n3n2)c1', 'Clc1ccc(Br)c(N2CSCC2)c1', 'Clc1ccc(Br)c(NN2C(=O)CSC2=O)c1', 'Clc1ccc(Br)c(N[C@H]2CCOC3(CC[NH2+]CC3)C2)c1', 'Clc1cc(F)c([N+](=O)[O-])cc1OCc1ccccc1[N+](=O)[O-]', 'Clc1cc(F)c(Cl)cc1C(=O)NNc1ccccc1', 'Clc1cc(F)c(NC2CC[NH+](CCCC(=O)[O-])CC2)cc1', 'Clc1cc(F)c(Nc2ncccc2C(=O)N2CCCCC2)cc1', 'Clc1cc(F)c(N2CCC2)cc1NCC(=O)N1CCOCC1', 'Clc1cc(F)c(N[C@H]2CC[NH2+]C[C@H]2Cc2ccccc2)cc1', 'Clc1cc(F)c(N/N=C/c2ccccc2O)cc1', 'Clc1cc(C[C@@H]2CCCN(S(=O)(=O)C[C@@H]3C=C(CN4CCOCC4)CO3)[C@@H]2c2ccccc2)ncn1', 'Clc1cc(C[C@@H]2SCCS2)ncn1', 'Clc1cccc(Br)c1S', 'Clc1cccc(Br)c1SCC(F)F', 'Clc1cccc(Br)c1S(=O)(=O)C1=c2ccccc2=[NH+]C1', 'Clc1cccc(Br)c1Sc1ccccc1', 'Clc1cccc(Br)c1S[C@@H]1CN(C(=O)c2cccc(F)c2)CC1', 'Clc1cccc(Br)c1S[C@H]1CCOC[C@@H]1C(=O)[O-]', 'Clc1ccsc1NSc1ccc(Br)cn1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1/N=N/c1ccc(Cl)cc1Br', 'Clc1ccsc1/N=N\\N1CCN(c2ccccc2F)CC1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1N[C@@H]1CCCN(c2ccccc2F)C1', 'Clc1ccsc1[C@@H]1CSCCS1', 'Clc1ccsc1[C@@H]1[C@H]2C(=O)N(c3cccc([N+](=O)[O-])c3)CC2=N1', 'Clc1ccsc1[C@@H]1[NH+]=c2ccccc2=[NH+][C@H]1COC(F)(F)F', 'Clc1nnn(C[C@@H](Br)CO)c1-c1ccc(Cl)cc1F', 'Clc1nnn(C[C@@H](O)CO)c1Br', 'Clc1nnn(C[C@@H](CO)C2CC2)c1Cl', 'Clc1cc(NSc2ccc(F)c([N+](=O)[O-])c2)ncn1', 'Clc1cc(Cl)ncc1Br', 'Clc1cc(Cl)ncc1Cc1ccccc1', 'Clc1cc(Cl)ncc1Oc1ccc2ccccc2c1', 'Clc1cc(Cl)ncc1N1CCc2ccccc21', 'Clc1cc(Cl)ncc1F', 'Clc1cc(Cl)ncc1[C@@H]1CCOC2(CCCC2)C1', 'Clc1cc(Cl)ncc1-c1cccc2c1OCCCO2', 'Clc1cc(Cl)ncc1Sc1ncccc1C(=O)[O-]', 'Clc1cc(Cl)ncc1Cl', 'Clc1cc(Cl)ncc1[O-]', 'Clc1cc(Cl)ncc1/C=C/C(=O)C(c1ccccc1)c1ccccc1', 'Clc1ccc(-c2csc3sc(=O)n(-c4ccccc4F)c23)cc1', 'Clc1ccc(-c2csc3sc(-c4ccoc4)cc23)cc1O', 'Clc1ccc(-c2cccs2)nc1SCc1ccccc1Cl', 'Clc1ccc(-c2cccnc2)cc1', 'Clc1ccc(-c2ccccc2)cc1C(=O)OCC(=O)Nc1cc(F)ccc1F', 'Clc1ccc(-c2cccs2)[nH]1', 'Clc1ccc(-c2cccs2)cc1', 'Clc1ccc(-c2cccs2)nc1N1CCc2ccccc21', 'Clc1ccc([C@@H](Cl)Cc2cccs2)cc1', 'Clc1ccc([C@@H](Cl)c2ccc3c(c2)OCCO3)cc1F', 'Clc1ccc([C@@H](Cl)[C@@H]2CCCOCC2)cc1', 'Clc1ccc([C@@H](Cl)[C@H]2CN3CCC[C@@H]3CO2)cc1', 'Clc1ccc([C@@H](Cl)[C@]2(F)CCc3ccccc32)cc1', 'Clc1ccc([C@@H](Cl)/C=C/Nc2ccc(Cl)cc2)cc1', 'Clc1cc(F)cc2nc(CN3CCCCCC3)nn12', 'Clc1ccccc1-c1n[nH]cc1COc1ccc(-c2ccccc2)nc1', 'Clc1ccccc1-c1n[nH]cc1CN1C[C@@H]2CC[C@@H](C1)C2=O', 'Clc1ccccc1-c1n[nH]c2c1[C@@H]1CC[C@H]1C[NH2+]C2', 'ClC[C@H](O)COc1cccc(F)c1Cl', 'ClC[C@H](O)c1ccc(Cl)cc1', 'ClC[C@H](O)[C@@H](n1cncn1)C1CC1', 'ClC[C@H](O)[C@H]1CN2CCC[C@@H]2CO1', 'ClC[C@H](O)[C@@]1(Cc2ccccn2)N2Cc3ccccc3[C@H](O)[C@H]12', 'Clc1nncc(F)c1C(F)(F)CC(F)(F)F', 'Clc1nncc(-c2c(F)cccc2F)n1', 'Clc1nncc(-c2ccc(F)cc2)n1', 'Clc1nncc(-c2cn3c(cnn3-c3cccc(F)c3F)c2)c1Cl', 'Clc1nncc(Cl)n1', 'Clc1nncc(Oc2ccc(F)cc2)c1N[C@H]1CCCOc2c(Cl)cccc21', 'Clc1cc(NSc2ccc(Cl)nc2Cl)ccc1[N+](=O)[O-]', 'Clc1cc(NSc2ccncn2)ncc1Cl', 'Clc1cc(Br)cnc1-c1ccc(F)cc1OCC(=O)N1CCO[C@H](CCC(F)(F)F)C1', 'Clc1cc(Br)cnc1-n1cccn1', 'Clc1cc(Br)ccc1OCC(=O)N[C@H]1c2ccccc2C1', 'Clc1cc(Br)c([C@H](O)c2ccccn2)cc1F', 'Clc1cc(Br)cnc1OC[C@H]1Cc2ccccc2S1', 'Clc1cc(Br)cnc1O/N=C1\\C(=O)N[C@@H](c2cc(Cl)c(O)cc2)[C@H]1O', 'Clc1cc(Br)cnc1Oc1cccc(Br)c1', 'Clc1cc(Br)cnc1O[C@@H]1CC[C@@H]([NH2+]C2CC2)C1', 'Clc1ccc(-c2cc[nH]n2)nc1[C@@H]1CCCN(C(=O)Nc2ccccc2)C1', 'Clc1ccc(-c2cc[nH]n2)nn1', 'Clc1nnnc(-c2cn(Cc3ccccc3)nn2)n1', 'Clc1nnnc(-n2cc(N3CCOCC3)nc2-c2ccccc2F)c1C1CC1', 'Clc1nnnc(-n2nc([O-])nc2-c2ccc(Cl)s2)n1', 'Clc1nnns1\n\n\n\n\n\n\n\n', 'ClC(=S)c1ccnc(C2CC2)n1', 'ClC([O-])=N/N=C/c1ccc(O)cc1O', 'ClCc1csc(OCC(=O)c2cccc(Cl)c2Cl)n1', 'ClCc1csc(OCc2ccccc2)n1', 'ClCc1csc(OC2CC2)n1', 'ClCc1csc(OC[C@@H]2C[C@H]2c2ccccc2)n1', 'ClCc1csc(OC[C@H]2CCCO2)n1', 'ClCc1csc2nc[nH]c12', 'ClCc1ccn(-c2ccc(C3CCCCC3)nn2)n1', 'ClCc1ccn([C@H](OC(F)F)C2CCCCC2)c1', 'ClCc1ccn([C@H]2CCC[NH2+]C2)c1', 'Clc1cccc([S@@]c2ccccn2)c1', 'Clc1cccc([S@@]c2nc3c(n2C)CCCC3)c1', 'Clc1cccc(Cn2ncc3c2NC23CC2)c1', 'Clc1cccc(/N=C/c2cccc(-n3cccn3)c2)c1', 'Clc1cccc(-c2cnc(C[NH+]3CCN(Cc4ccc(F)cc4F)C3)o2)c1', 'Clc1cccc(-c2noc(CSc3ncnc4[nH]ncc34)n2)c1', 'Clc1cccc([S@@](=O)CC(F)F)c1O', 'Clc1ccccc1Nc1nc(Cl)nc(NN2CCOCC2)c(=O)c1C#N', 'Clc1ccccc1Nc1ncnc2c1CCCC2', 'Clc1ccccc1Nc1ncccc1Cl', 'Clc1ccccc1Nc1nc2sccn2c1Br', 'Clc1cccc(/N=C/c2ccccc2)c1', 'Clc1cccc(/N=N/c2cccc(Oc3ccccc3)c2)c1', 'Clc1cccc(/N=C/c2ccc(-c3ccccc3)cc2)c1', 'Clc1cccc(/N=C(\\[O-])NCc2ccco2)c1O', 'Clc1cccc(/N=C\\c2ccccc2)c1', 'Clc1cccc(/N=C2\\O[C@H]3CCCC[C@@H]32)c1F', 'Clc1cccc(Nc2cccc(COc3ccccc3)c2)c1', 'Clc1cccc(CNC[C@H]2CCC[C@@H]2n2cccn2)c1', 'Clc1cccc(CN(Cc2nc3ccccc3s2)C2CC2)c1', 'Clc1cccc(CNc2ncnc3sc(-c4ccccc4)cc23)c1', 'Clc1cccc(CN2CC[NH+](CC[C@H]3CC[NH+](Cc4ccc([N+](=O)[O-])cc4)CC3)C2)c1', 'Clc1cccc2ncn(C[C@H](O)c3ccco3)c12', 'Clc1cccc2ncnc(N3CCOC4(CCCC4)C3)c12', 'Clc1ccc(Br)c(F)c1NC1[C@@H]2CC[C@@H]1C2', 'Clc1ccsc1[C@H]1COc2ccccc21', 'Clc1ccsc1[C@H]1Oc2ccccc2[C@@H]1Nc1cccc([N+](=O)[O-])c1', 'Clc1ccsc1[C@H]1N=C(C[NH+]2CCN(c3ccc(Cl)cc3)CC2)C1', 'Clc1ccsc1[C@H]1n2cccc2C2=NC1=NC2', 'Clc1ccsc1[C@H]1[NH+]=c2ccccc2=[NH+]1', 'ClC(=S)[C@@H](c1ccccc1)N1CCCc2ccccc21', 'ClC(=S)[C@@H]1CCCS1', 'ClCc1csc(-c2ccccc2)n1', 'ClCc1csc(N2CCNC2=O)n1', 'ClCc1csc(N2[C@H](C(=O)[O-])CCOC2)n1', 'ClCc1cncc(C(=O)OCc2nc[nH]n2)c1', 'ClCc1cnccc1CCc1nc2c(c(=O)[nH]1)S2', 'ClCc1cncc2[nH]nnc12', 'Clc1nnc(Cl)n2ncc(-c3ccc(F)cc3F)c12', 'Clc1nnc(Br)s1', 'Clc1nnc(Br)cn1', 'Clc1nnnc(Cl)n1', 'ClC[C@@H]1Oc2ccccc2[C@@H]1Cc1c(F)cccc1Cl', 'ClC[C@@H]1Oc2ccccc2C[C@@H]1O', 'ClC[C@@H]1Oc2ccccc2C(=O)N1c1ccccc1C(F)(F)F', 'ClC[C@@H]1Oc2cc(F)ccc2C[C@@H]1F', 'ClC[C@@H]1Oc2ccccc2NC1', 'Clc1nncc(F)c1C(F)(F)F', 'Clc1nncc(F)c1I', 'Clc1nncc(F)c1N[C@H]1CCc2c(Br)cccc21', 'Clc1nncc(F)c1F', 'Clc1nncc(F)c1-c1ccc(Cn2ccccc2=O)cc1', 'Clc1nncc(F)c1Br', 'Clc1cc(/C=N\\c2ccc(Cl)cc2)c(F)cc1[N+](=O)[O-]', 'Clc1cc(/C=N\\OC(=O)COc2ccccc2Cl)c2c(c1)OCO2', 'Clc1cc(/C=N\\O)ccc1F', 'Clc1cc(/C=N\\Oc2cccc(Br)c2)cc([O-])c1Cl', 'Clc1cc(/C=N\\O[C@@H]2CCC[C@@H]2[C@@H]2CCCO2)ccc1O', 'Clc1cc(/C=N\\O[C@H]2c3ccccc3O2)ccc1[O-]', 'ClC[C@@H]1Oc2ccc(F)cc2[C@@H]1C1CCCCCC1', 'ClC[C@@H]1Oc2ccccc2C[NH+]1CCC1CC1', 'Clc1cccc2ncn(CCCSc3ccccn3)c12', 'Clc1cccc2ncn([C@@H](CO)C(=O)N3CCCC3)c12', 'Clc1cccc2nn(S[C@@H](CO)c3ccccc3)nc12', 'Clc1ccccc1Oc1cnc(N2CCOCC2)c2cc(Br)ccc12', 'Clc1ccccc1Oc1cncc(-c2ccccc2)[nH+]1', 'Clc1ccccc1Oc1cnc2ccccc2c1OC[C@H]1CCCO1', 'Clc1ccccc1N1N(C2(NS(=O)(=O)c3ccccc3)COC2)SC1=O', 'Clc1ccccc1NS(=O)(=O)c1ccc(Oc2ccccc2)cc1', 'Clc1ccccc1N1N=CC[C@@H]1C(=O)Nc1cccc2ccccc12', 'Clc1ccccc1Cc1cc(F)ccc1O', 'Clc1ccccc1Cn1cnc2cc(F)ccc21', 'Clc1ccccc1CI\n', 'Clc1ccccc1Cc1ccc(Cl)c(F)c1', 'Clc1ccccc1Cc1nc(Cn2ccccc2=O)no1', 'Clc1ccccc1NS(=O)(=O)c1cccc(S(=O)(=O)N2CCCCC2)c1', 'Clc1ccccc1NS(=O)(=O)c1ccc(F)cc1', 'Clc1ccccc1N1Nc2ccccc2C(=O)N1Cc1cccc(F)c1', 'Clc1ccccc1OCc1ccc(CN2CCCn3cc(O)cc3C2)cc1', 'Clc1cccc(Nc2ccc(Cl)c(C(=O)Nc3nnc(-c4ccccc4)o3)n2)c1', 'Clc1cccc(Nc2cnc(-c3cnn(-c4nc(C(F)(F)F)ccc4Cl)c3)[nH]c2=O)c1', 'Clc1cccc([S@@]c2c[nH]nc2)c1', 'Clc1cccc([S@@]c2cc(I)cnc2Cl)c1', 'Clc1cccc([S@@]c2cnccc2[O-])c1', 'Clc1cccc(Nc2cc(F)cc(Cl)c2F)c1', 'Clc1cccc(S[C@H](C(=O)[O-])C2CCOCC2)c1', 'Clc1cccc(S[C@H]2CCOc3ccccc32)c1', 'Clc1cccc(N2CCC([NH+]3CCC[C@H]C3[C@@H]3CCCO3)C2)c1', 'Clc1cccc(N2N=C(CC3=c4ccccc4=[NH+]C3)CC2)c1', 'Clc1cc(COCCO)cs1', 'Clc1cc(COC(=O)c2ccco2)ccc1[N+](=O)[O-]', 'Clc1cc(COCc2ccc[nH]2)ncn1', 'Clc1cc(COC2=N[C@@H]3CS(=O)(=O)C[C@@H]3N(c3ccncc3)C2)ccc1Cl', 'Clc1cc(COCOc2ccc(Cl)cc2)ncn1', 'Clc1cc(COC[C@@H]2CCCO2)ccc1NCc1coc(-c2ccccc2)n1', 'Clc1cc(COC[C@H]2CCCN(c3ccccc3F)C2)sc1NC1CC1', 'Clc1cc(F)c([C@H](Oc2ccccc2)[C@@]2(C(F)(F)F)NC(=S)N=C2C(=O)[O-])cc1', 'Clc1cc(F)c([C@H]2CC(=O)N2c2ccc(Cl)cc2Cl)cc1', 'Clc1cccc2cccnc12', 'Clc1cccc2c(Oc1[O-])nc(Cn1cccn1)nn2C1CC1', 'Clc1nnnn1c1nc(-c2ccco2)cs1', 'Clc1nnnn1c1nnnn1C1CCCC1', 'Clc1nnnn1-c1cc([N+](=O)[O-])ccc1O', 'Clc1nnnn1-c1cc(F)ccc1Br', 'Clc1nnnn1-c1cc(S(=O)(=O)N2CCCC2)ccc1Cl', 'Clc1nnnn1-c1cc(Cl)cc([N+](=O)[O-])c1', 'Clc1nnnn1-c1cc(Br)cc(Br)c1', 'Clc1nnnn1c1cc(Cl)c(Cl)cc1Cl', 'Clc1nnnn1c1cccc(OCCO)c1', 'Clc1nnnn1c1c([N+](=O)[O-])cccc1OCC(F)F', 'Clc1nnnn1c1c(C(F)(F)F)ccc2c1OCO2', 'Clc1nnnn1c1c(O)cccc1N1CCCC1', 'Clc1nnnn1c1c(NC2CCN(c3ccccc3)CC2)ncn1-c1ccccc1', 'Clc1nnnn1c1c(F)cc(Oc2ccc(F)cc2)c(Cl)c1O', 'Clc1nnnn1c1c(Cl)cccc1N1CC[NH2+]CC1', 'Clc1nnnc(Oc2c(Cl)cccc2OCC2CC2)c1Cl', 'Clc1nnnc(O[C@@H](C(=O)c2ccccc2)c2ccccc2)c1N', 'Clc1nnnc(O[C@@H]2CCCCCC[C@@H]2O)n1', 'Clc1nnn(-c2ncn[nH]c2=O)n1', 'Clc1nnn(-c2ncn[nH]2)c1-c1ccc(Cl)cc1', 'Clc1nnnn1-c1cc(S(=O)(=O)N2CCCC2)ccc1Cl', 'Clc1nnnn1-c1c(-c2ccccc2)c[nH]c1CSc1nc2ccccc2n1CC(F)F', 'Clc1nnnn1-c1cc(F)ccc1Cl', 'Clc1nnnn1-c1cc(Cl)ccc1Cl', 'Clc1nnnn1/C(CCC(F)(F)F)N[C@H]1CC[C@@H](SCC(F)(F)F)C1', 'Clc1nnnn1/C(\\[O-])c1ccccc1C(F)(F)F', 'Clc1nnnn1/C(c1cccs1)[C@H]1C[C@H]1c1cccc(Cl)c1Cl', 'Clc1nnnn1/C(Cl)Cc1cccc(Br)c1', 'Clc1nnnn1/C([O-])=C/c1ccc2ccccc2c1', 'Clc1nnnn1/C(/[O-])CC(=O)Nc1cccc(C(F)(F)F)c1', 'Clc1cc(Br)ccc1C[NH+]1CCC[C@@H](CCO)C1', 'Clc1cc(Br)ccc1O[C@@H]1CCOC2(CCC2)C1', 'Clc1cc(Br)ccc1NC1(F)CCCCC1', 'Clc1cc(Br)ccc1-c1ccccc1F', 'Clc1cc(Br)ccc1Sc1ccccc1', 'Clc1cc(Br)ccc1Cl', 'Clc1cc(Br)ccc1[C@H]1C[NH+]2CCCC[C@H]12', 'Clc1nnc(Cl)nc1F', 'Clc1ccccc1NS(=O)(=O)c1ccc([N+](=O)[O-])cc1', 'Clc1ccccc1NS(=O)(=O)c1ccc(C(=O)N2CCOCC2)cc1', 'Clc1ccccc1-c1ncc(Cl)s1', 'Clc1ccccc1-c1nccn1C1CCN(S(=O)(=O)c2cccc(Cl)c2F)CC1', 'Clc1ccccc1-c1nccs1', 'Clc1ccccc1-c1ncc[nH]1', 'Clc1ccccc1N1N(CC(=O)Nc2cccc(Cl)c2)CC1', 'Clc1ccccc1N1N(Cc2ccccc2F)c2ccccc21', 'Clc1ccccc1N1N(C2CCCCCCC2)c2ccccc21', 'Clc1ccccc1N1N(C[C@@H]2CCCO2)C=C1c1cccc(Cl)c1', 'Clc1ccccc1-c1noc2c1N=C(SCc1ccccc1)C(=O)C[C@@H]2c1cccc(F)c1', 'Clc1cccc([C@@H](CCc2c[nH]c3ccccc23)N=C2CCOCC2)c1', 'Clc1cccc([C@@H](c2cccs2)COc2ccc([N+](=O)[O-])cc2)c1O', 'Clc1cccc([C@@H](N[C@H]2CCCc3ccc(F)cc32)C2CC2)c1', 'Clc1cccc([C@@H](Sc2ccc(Cl)cc2)c2ccc(F)cc2)c1', 'Clc1cccc([C@@H]([NH2+]Cc2nccs2)C2CC2)c1', 'Clc1ccc(-c2cccnc2)cc1CN1CCN(c2ccccc2N)CC1', 'Clc1ccc(-c2ccccc2C(F)(F)F)cc1', 'Clc1ccc(-c2ccccc2)cc1NC[C@H]1CCOC1', 'Clc1ccc(-c2ccccc2[N-]S(=O)(=O)c2cc(Br)ccc2F)cc1', 'Clc1ccc(-c2ccccc2N2CCN(c3ccc(Cl)c(Cl)c3)CC2)cc1', 'Clc1ccc(-c2ccccc2Cl)cc1NCC(=O)[O-]', 'Clc1ccc(-c2cc(Br)ccc2N2CCOCC2)cn1', 'Clc1ccc(-c2cc(-c3cc[nH]n3)ncn2)cc1', 'Clc1ccc(-c2cc(-n3ccnc3)cn3cnnc23)cc1', 'Clc1ccc(-c2cc[nH]n2)o1', 'Clc1ccc(Br)cc1\n\n', 'Clc1ccc(Br)cc1Cl\n', 'Clc1ccc(Br)cc1/C=C(/C#N)C(=O)N[C@@H]1CCS(=O)(=O)C1', 'Clc1ccc(Br)cc1/N=C/c1ccc(F)cc1', 'Clc1ccc(Br)cc1\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n', 'Clc1ccc(Br)cc1[C@@H](O)COC(F)(F)F', 'Clc1ccc(Br)cc1[C@@H]1CCCN1C(=O)c1ccc2c(c1)CCC2', 'Clc1ccc(Br)cc1-c1ccc(CO)o1', 'Clc1ccc(Br)cc1NC(=O)CN1CCCSc2ccccc21', 'Clc1ccc(Br)cc1N(Cc1cccc(O)c1)c1cccc[n+]1[O-]', 'Clc1ccc(Br)cc1N1CCC[C@H]1C[NH+]1CCCC1', 'Clc1ccc(-c2nonc2NC2=N[C@H]3C4CCCCC43)[C@H]2C1', 'Clc1ccc(-c2nonc2N2CCC(Oc3ccc(F)cc3)CC2)cc1', 'Clc1ccccc1Oc1cnc(Cc2ccccc2)n(-c2ccccc2)c1=O', 'Clc1ccccc1Oc1cnc(Oc2ccc(Br)cc2F)cn1', 'Clc1ccccc1Oc1cnc(N2CCOCC2)nc1', 'Clc1ccccc1Oc1cnc(I)cc1O', 'Clc1ccccc1Oc1cnc(F)cc1', 'Clc1ccccc1Oc1cnc([C@@H]2CCOC3(CCCCC3)C2)nc1N', 'Clc1ccccc1Oc1cnc(-c2ccccc2Cl)n1-c1ccccc1', 'Clc1ccccc1Oc1cnc(Cl)c(Cl)c1', 'Clc1ccccc1Oc1cnc(Br)cn1', 'Clc1ccccc1Oc1cnc(/N=C/c2ccccc2Cl)cc1', 'Clc1ccccc1C[NH+](C1CC1)[C@@H]1CCC[C@H](C(F)(F)F)C1', 'Clc1ccccc1C[NH+]1CCC(CNc2ccc(Cl)cc2[N+](=O)[O-])CC1', 'Clc1cccc2[nH]nnc12', 'Clc1cccc2[nH]nnc12\n', 'Clc1cccc2[nH]nnc12\n\n', 'Clc1cccc2[nH]nnc12\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n', 'Clc1cccc2[nH]nc(NC[C@@](CO)[N+](=O)[O-])c12', 'Clc1cccc2[nH]nc(N)c12', 'Clc1cccc2[nH]nc(N[C@H]3CCCC[C@@H]3[NH2+]C[C@@H](O)c3ccccn3)c12', 'Clc1cccc2[nH]nc(N/N=C/c3ccccc3)c12', 'Clc1ccccc1NS(=O)(=O)N1CCC(CN2CCCCCC2)CC1', 'Clc1ccccc1NS(=O)(=O)c1ccc(Br)cc1', 'Clc1cccc(-c2ncco2)c1N[C@H]1CCCc2ccccc21', 'Clc1cccc(-c2n[n-]c(-c3ccccc3)n2)c1', 'Clc1cccc(-c2nnc(NC[C@@H]3CCCO3)[nH]2)c1', 'Clc1cccc(-c2n[nH]c(CSc3nncc4ccc(Br)cc34)n2)c1', 'Clc1cccc2ncnc(NC[C@@H](O)c3ccoc3)c12', 'Clc1cccc2ncncc12', 'Clc1cccc2ncnc2c1/N=C\\c1ccc(COC2CCCC2)cc1', 'Clc1ccc([C@@H](N[C@@H](C#N)Cc2nc3ccccc3[nH]2)C2CC2)cc1', 'Clc1ccc([C@@H](Br)c2ccc(Cl)c(F)c2F)cc1', 'Clc1ccc([C@@H](NCCC2CCOCC2)c2ccccc2)cc1', 'Clc1ccc([C@@H](NC[C@H](O)c2ccc[nH]2)c2ccccc2)cc1', 'Clc1ccc([C@@H](NCc2ccccn2)Cc2cccnc2)cc1', 'Clc1ccc([C@@H](NC2COC2)C(F)(F)F)cc1', 'Clc1ccc([C@@H](Br)CCCO)cc1', 'Clc1ccc([C@@H](Br)C(F)(F)F)c(F)c1', 'Clc1ccc([C@@H](Br)Cc2cccnc2)cc1', 'Clc1ccc([C@@H](Br)COc2ccccc2)cc1', 'Clc1ccc([C@@H](Br)C[C@@H]2CCOC3(CCCC3)C2)cc1', 'Clc1ccc([C@@H](Br)Cn2ccnc2)cc1', 'Clc1ccc([C@@H](Br)CSCc2ccccc2)cc1', 'Clc1ccc([C@@H](Br)C[C@H](c2ccc(Cl)cc2)[NH+](C)C)cc1', 'Clc1ccc([C@@H](Br)C[NH+]2CCC(C(N)=O)CC2)s1', 'Clc1ccc(Br)c(Cl)c1C(=O)Nc1ccc(C(F)(F)F)cc1', 'Clc1ccc(Br)c(-c2ccco2)c1N', 'Clc1ccc(Br)c(-n2cccn2)c1', 'Clc1ccc(Br)c(CCC(=O)N2[C@H](c3ccccc3)OC2(C)O)c1', 'Clc1ccc(Br)c(C(=O)/C=C/C2=CCCCC2)c1', 'Clc1ccc(Br)c(CN2CCc3ccccc3C2)c1', 'Clc1ccc(Br)c(C[C@@H]2CCC[NH2+]2)c1', 'Clc1ccc(Br)c(Cn2ccc3ccc(O)cc32)c1', 'Clc1ccc(Br)c(CCl)c1', 'Clc1ccc(Br)c(C[NH+]2CCN([C@@H](C(N)=O)c3ccccc3)C2)c1', 'Clc1ccc(Br)c(C[NH2+]C[C@@H]2CCN(c3cccc(F)c3)C2)c1', 'Clc1cccc([S@@]Cc2nc(-c3ccccc3)no2)c1', 'Clc1cccc([S@@](=O)CCCC[NH+](C2CC2)[C@@H]2CCCO2)c1', 'Clc1cc(CCl)c[nH]c1=O', 'Clc1cc(CCl)c(F)cc1F', 'Clc1cc(CCl)cc(I)c1O', 'Clc1ccc(Cc2cccc(O)c2)nc1', 'Clc1ccc(COc2ccc(F)cc2)cc1F', 'Clc1ccc(Cc2ccccc2)c(F)c1', 'Clc1ccc(Cc2noc(-c3ccc([N+](=O)[O-])cc3)n2)cc1', 'Clc1ccc(COCC[C@@H]2COc3ccccc32)cc1', 'Clc1ccc(COCc2cccc(C(Br)(F)F)c2)cc1', 'Clc1ccc(COC2CC[NH+](C[C@@H]3CCCO3)CC2)cc1', 'Clc1ccc(COCOc2ccc3ccccc3c2)cc1Cl', 'Clc1ccc(COC[C@@H](O)c2ccccc2)cc1', 'Clc1ccc(COC[C@H]([NH2+]Cc2ncc[nH]2)c2ccccc2)cc1', 'Clc1ccc(Cc2csc(-c3c[nH]nc3N3CCCC3)n2)cc1', 'Clc1ccc(Cc2c[nH]nc2-c2ccccc2)c(Cl)c1', 'Clc1ccc(Cc2cccc(CNC(=O)C3CC3)c2)cc1', 'Clc1ccc(C2C(C(=O)NCc3cccc(C(F)(F)F)c3)CC2)cc1', 'Clc1ccc(C2CC2)cc1NCc1nc(C2CC2)no1', 'Clc1ccc(C2CN2)cc1', 'Clc1ccc(C2C[C@](NC3CC3)/C(=C/Cl)C2)cc1', 'Clc1ccc(C(CCCO)CBr)cc1', 'Clc1ccc(C(CO)OC(F)F)cc1', 'Clc1ccc(C(C)C2CCN(C(=O)[C@H]3COc4ccccc4O3)CC2)cc1', 'Clc1ccc(CNS(=O)(=O)c2ccc([N+](=O)[O-])cc2)cc1', 'Clc1ccc(CNSc2ncnc3c2C[NH+](CC(F)(F)F)C3)cc1', 'Clc1ccc(Cc2noc(-c3ccc(Cl)o3)n2)c(F)c1', 'Clc1ccc(Cc2nc(C(F)(F)F)cs2)cc1', 'Clc1ccc(Cc2nnc(NC[C@H]3CCCO3)o2)cc1', 'Clc1ccc(C(N)=O)cc1', 'Clc1ccc(C2(CO)CC2C[NH2+]Cc2ccsc2)cc1', 'Clc1ccc(C2(c3ccccc3F)C[NH2+]C2)cc1', 'Clc1ccc(C2(OC[C@@H]3CCN(C4CCCCC4)C3)CC2)cc1', 'Clc1ccc(C2(NCC(=O)[O-])C[C@@H]2C2CCOCC2)cc1', 'Clc1ccc(C2(F)CC2)cc1', 'Clc1ccc(C2(Sc3ncnc4ccccc34)CCC[C@@H]2CO)cc1', 'Clc1ccc(C2(Cl)CCC2)cc1', 'Clc1ccc(-c2cccnc2Cl)cc1', 'Clc1ccc(-c2cccnc2S(=O)(=O)N2CCCCC2)cc1', 'Clc1ccc(-c2cccnc2N2CCSCC2)cc1', 'Clc1ccc(-c2cccnc2)cc1N1CCOCC1', 'Clc1ccc(-c2cc(F)ccc2F)cc1', 'Clc1ccc(-c2cc[nH]n2)o1', 'Clc1ccc(-c2cc(OC[C@@H]3CCCCO3)co2)cc1', 'Clc1ccc(-c2cc(O)n[nH]2)cc1', 'Clc1ccc(-c2cc(O[C@H](Cc3cccc(F)c3)c3ccccc3)n[nH]2)cc1', 'Clc1ccc(-c2cc[nH]n2)nc1Sc1ccccc1', 'Clc1ccc(-c2cccs2)c(NC[C@H]2[C@@H]3CC[C@@H]2C3)c1', 'Clc1ccc(-c2cccs2)cc1OCCC1(F)CCCC1', 'Clc1ccc(-c2cccs2)c2nnc(-c3ccccc3Cl)nc12', 'Clc1ccc(-c2nnnn2CCNc2ccc(F)cc2Cl)cc1', 'Clc1ccc(-c2nnnn2Cc2ccccc2Cl)cc1', 'Clc1ccc(-c2nnnn2C2CC2)cc1', 'Clc1ccc(-c2nnnn2C[C@@H]2CC3(CCOCC3)C2)cc1', 'Clc1ccc(-c2nnnn2C[C@H](O)c2ccccc2)cc1', 'Clc1ccc(-c2nnnn2C[C@]2(O)CCOC2)cc1', 'Clc1ccc(-c2n[nH]c3[nH]c4ccccc4c23)cc1', 'Clc1ccc(-c2n[nH]c3c2[nH]c(=S)n3C(F)(F)F)cc1', 'Clc1ccc(-c2n[nH]c3ncnn23)cc1Cl', 'Clc1ccc(-c2nnnn2-c2ccccc2)cc1Cl', 'Clc1ccc(-c2nnnn2Cl)cc1', 'Clc1ccc(-c2nonc2[S-])cc1', 'Clc1ccc(-c2cc(SCC(=O)NCC3CC3)ccc2Cl)cc1', 'Clc1ccc(-c2cc(S(=O)(=O)C[C@@H](O)c3ccncc3)no2)cc1', 'Clc1ccc(-c2cc(Sc3nc4ccccc4[nH]3)co2)cc1', 'Clc1ccc(-c2nnnn2[C@@H](c2cccs2)c2ccccc2)cc1', 'Clc1ccc(-c2nnnn2[C@@H]2CCC[NH+](Cc3cc(F)ccc3F)C2)cc1', 'Clc1ccc([C@@H](OCC(F)F)c2cccnc2)C[C@@H]1CC(=O)Nc1ccc(F)cc1', 'Clc1ccc([C@@H](OC(=O)C2Oc3ccccc3F)C2)cc1', 'Clc1ccc([C@@H](OC)C(=O)N2CCC[C@@H](Cn3cc4ccccc4n3)C2)cc1', 'Clc1ccc([C@@H](OCc2ccccc2)c2ccccc2)cn1', 'Clc1ccc([C@@H](OC2CCCC2)C(F)(F)F)o1', 'Clc1ccc([C@@H](OC[C@@H](O)C2CC2)N2CCN(c3ccccc3)CC2)cc1', 'Clc1ccc([C@@H](OC[C@H]2CCC[NH2+]C2)[NH+]2CCCC2)cc1', 'Clc1ccc(Cc2cc(F)c(F)c(Cl)c2)cc1Br', 'Clc1ccc(Cc2cc[nH]c(=O)c2)cc1', 'Clc1ccc(Cc2ccc(OCc3ccc(Cl)cc3Cl)o2)cc1', 'Clc1ccc(C(F)(F)F)cc1F', 'Clc1ccc(C(N)=O)cc1', 'Clc1ccc(COc2nc(Cl)ccc2Oc2ccccc2F)cc1', 'Clc1ccc(C(F)(F)F)cc1Oc1ccccc1Br', 'Clc1ccc(CO)cc1O', 'Clc1ccc(CO)nc1NCc1ccc(N2CCC2)nc1', 'Clc1ccc(CN2CCO[C@@H](c3cccc(C(F)(F)F)c3)C2)cc1Cl', 'Clc1ccc(CN2N=C3SCC=C3SC2)cc1N1CCCC1', 'Clc1ccc(C[NH+]2CCC[C@]2(C(=O)[O-])[C@@H]2CCOC3(CCOCC3)C2)cc1', 'Clc1ccc(C[NH+]2[C@H]3CCCC[C@H]3C[C@@H]2C2CCCC2)cc1', 'Clc1ccc(C(N)=O)cc1OCc1nnn[n-]1', 'Clc1ccc(C(N)=NC2CCCS2)cc1', 'Clc1ccc(C(N)=[NH2+])cc1F', 'Clc1ccc(Br)cc1\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n', 'Clc1ccc(Br)cc1-n1cccc1C(=O)N1CCCC[C@H]1CO', 'Clc1ccc(Br)cc1\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n', 'Clc1ccc(Br)cc1-c1ccccc1F', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n', 'Clc1ccc(Br)cc1/N=C1/NNc2ccc([N+](=O)[O-])cc21', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1-n1cccc1Cn1ccccc1=O', 'Clc1ccc(Br)cc1-n1nnnc1SCc1ncc(-c2ccccc2)o1', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1[C@@H]1COC2(CCCC2)C1', 'Clc1ccc(Br)cc1[C@@H]1OCCc2ccccc21', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1S(=O)(=O)Nc1ccc(Cl)c(F)c1', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1/C=N/N1CC[NH2+]CC1', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1-c1ccccc1Cl', 'Clc1ccc(Br)cc1-c1cnn(-c2ccc(F)cc2)n1', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Sc1cccc(F)c1', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1S[C@H](C1=c2ccccc2=[NH+]C2)C=C[C@@H]12', 'Clc1ccc(Br)cc1S[C@H]1CCC(=O)NC1=O', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1-n1cccn1', 'Clc1ccc(Br)cc1-n1cnc2ccccc21', 'Clc1ccc(Br)cc1OS(=O)(=O)c1cccc(Br)c1', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1OCCN1CCN(c2ccccc2)CC1', 'Clc1ccc(Br)cc1OC(=O)c1cccnc1', 'Clc1ccc(Br)cc1OCc1cccc(F)c1', 'Clc1ccc(Br)cc1OC1CC[NH2+]CC1', 'Clc1ccc(Br)cc1OC[C@H]1CCCO1', 'Clc1ccc(Br)cc1OC/C=C/c1ccccc1', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1-c1cc(F)ccc1Cc1ccc(Br)cc1', 'Clc1ccc(Br)cc1-c1ccccc1', 'Clc1ccc(Br)cc1-c1cc2ccccc2cc1F', 'Clc1ccc(Br)cc1-c1ccncc1', 'Clc1ccc(Br)cc1-c1ccsc1', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1[C@@H](CO)N[C@@H](CO)c1ccco1', 'Clc1ccc(Br)cc1[C@@H](c1ccccc1)S(=O)(=O)c1cccc([N+](=O)[O-])c1', 'Clc1ccc(Br)cc1[C@@H](O)[C@@H]1CCOC2(CCC2)C1', 'Clc1ccc(Br)cc1[C@@H](Cl)c1ccccc1', 'Clc1ccc(Br)cc1[C@@H]([O-])CSc1ccccc1', 'Clc1ccc(Br)cc1[C@@H]([NH3+])COOCc1cccc(F)c1', 'Clc1ccc(Br)cc1[C@@H]([NH2+]Cc1ccccc1)C1CC1', 'Clc1ccc(Br)c(Cl)c1Cl', 'Clc1ccc(Br)c(Oc2cccc(OC[C@H]3CCC(=O)N3)c2)c1', 'Clc1ccc(Br)c(Cl)c1', 'Clc1ccc(Br)c(Cl)c1Cn1cccn1', 'Clc1ccc(Br)c(Cl)c1OCCN1CCN(c2cccc[nH+]2)CC1', 'Clc1ccc(Br)c(Cl)c1N1CCC[C@H]([NH3+])C1', 'Clc1ccc(Br)c(Cl)c1[C@@H](CO)NC1(N2CCOCC2)CCCC1', 'Clc1ccc(Br)c(Cl)c1Cl', 'Clc1ccc(Br)cc1/N=C/c1cc(C(F)(F)F)ccc1SC1CC1', 'Clc1ccc(Br)cc1/N=N\\c1ccccc1Br', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1/N=C/c1ccc2c(c1)OCO2', 'Clc1ccc(Br)cc1/N=C(\\CCn1cccn1)N1CC[NH+]2CCCC[C@@H]12', 'Clc1ccc(Br)cc1/N=C\\NC1CCN(c2ccc(F)c(F)c2)CC1', 'Clc1ccc(Br)c(-n2ncc(C(F)(F)F)c2Cl)c1', 'Clc1ccc(Br)c([C@H]([NH2+]Cc2cccc(Cl)c2)[C@H]2CCSC2)c1', 'Clc1ccc(Br)c([C@H]2CCC[NH+]2Cc2ccccc2)c1', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1-n1ncc2ccccc21', 'Clc1ccc(Br)cc1-n1nnc(-c2cccs2)n1', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1S(=O)(=O)N1CCOCC1', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1-n1cn[nH]c1=O', 'Clc1ccc(Br)cc1-n1cnc2c(F)cccc21', 'Clc1ccc(Br)cc1-n1cnnc1', 'Clc1ccc(Br)cc1Sc1ccccc1', 'Clc1ccc(Br)cc1Sc1nc2ccc(F)cc2n1C1CC1', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1N(Cc1ccns1)C(=O)c1cccc(Br)c1', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1N(CCS(=O)(=O)c1ccc(Cl)c(Cl)c1)[C@H]1CCS1(=O)[O-]', 'Clc1ccc(Br)cc1N(C(=O)c1ccc(F)cc1)[C@@H]1CSCS1', 'Clc1ccc(Br)cc1N(Cc1ccc(Br)o1)C1CC1', 'Clc1ccc(Br)cc1N(C1CCOCC1)C1CCCCC1', 'Clc1ccc(Br)cc1N(CN1CCOCC1)C1CC1', 'Clc1ccc(Br)cc1N(C[C@@H]1CCCO1)[C@H]1C[C@@H]1c1cc(Br)ccc1O', 'Clc1ccc(Br)cc1N(C[C@H]1CCCO1)c1ccccc1', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1SCCC(F)(F)F', 'Clc1ccc(Br)cc1SCc1ccc(F)cc1', 'Clc1ccc(Br)cc1SC1CC[NH2+]CC1', 'Clc1ccc(Br)cc1SC[C@H]1Cc2ncccc2-c2ccccc21', 'Clc1ccc(Br)cc1SC/C=C\\c1cnc(Oc2ccccc2)c([O-])c1C(=O)[O-]', 'Clc1ccc(Br)cc1O[C@H](c1ccccc1)[C@@H]1CCCO1', 'Clc1ccc(Br)cc1O[C@H]1C=CCCC1', 'Clc1ccc(Br)c(F)c1F', 'Clc1ccc(Br)c(/N=N/c2ccccc2)c1', 'Clc1ccc(Br)c(O)c1O[C@@H]1CCCC[C@@H]1OCCc1ccccc1', 'Clc1ccc(Br)c([C@@H](CO)Nc2ccc(F)cc2)n1', 'Clc1ccc(Br)c([C@@H](O)c2cccnc2)c1', 'Clc1ccc(Br)c([C@@H]([NH2+]Cc2cnc3cc[nH]c3c2)C2CC2)c1', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1-c1cn(-c2ccccc2F)[nH]c1=O', 'Clc1ccc(Br)cc1-c1cncnc1', 'Clc1ccc(Br)cc1-c1cnn(-c2ccc(F)cc2)n1', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n', 'Clc1ccc(Br)cc1-c1cnc(C2CC2)nn1', 'Clc1ccc(Br)cc1-c1cncnc1', 'Clc1ccc(Br)cc1-c1cnccn1', 'Clc1ccc(Br)cc1-c1cnc2ccccc2n1', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccccc1', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O[C@@H](c1ccccc1)C1CC1', 'Clc1ccc(Br)cc1O[C@@H]1CCOC2(CCC2)O1', 'Clc1ccc(Br)cc1/C=N/NC(=O)C(c1ccccc1)c1ccccc1', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1/C=N/N=C1\\S[C@@H]2CS(=O)(=O)C[C@H]12', 'Clc1ccc(Br)cc1/C=N\\O[C@@H]1CCCO[C@@H]1c1ccccc1', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1-n1nc(C(F)(F)F)nc1N1CCOCC1', 'Clc1ccc(Br)cc1-n1ncn(CN2CCOCC2)c1=O', 'Clc1ccc(Br)cc1-n1ncc2sccc21', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1COC(=O)[C@H]1C[C@H]1c1ccc(Cl)cc1', 'Clc1ccc(Br)cc1CO/[NH+]=C(\\N)C#N', 'Clc1ccc(Br)cc1COc1ccc(Br)cc1', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n', 'Clc1ccc(Br)cc1-c1cc(=O)oc2ccccc12', 'Clc1ccc(Br)cc1-c1cc(F)ccc1F', 'Clc1ccc(Br)cc1-c1cc(-c2cccnc2)ncn1', 'Clc1ccc(Br)cc1-c1cc(Cl)cc2ccccc12', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1OS(=O)(=O)c1cc(F)ccc1F', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1S(=O)(=O)N[C@@H]1CCSc2ccccc21', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n', 'Clc1ccc(Br)cc1[C@@H]1O[C@H](CO)[C@H](O)[C@H]1Cc1ccccc1', 'Clc1ccc(Br)cc1[C@@H]1Oc2ccccc2C(=O)N1CCCC(F)(F)F', 'Clc1ccc(Br)cc1[C@@H]1O[C@@H]2CCC[C@H]2[C@@H]1[NH3+]', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1OS(=O)(=O)c1c(Cl)cccc1Cl', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Sc1c(F)cc(F)cc1F', 'Clc1ccc(Br)cc1Sc1ccccc1', 'Clc1ccc(Br)cc1-n1cc[nH+]c1CN1C[C@@H]2CC[C@H](C1)C2', 'Clc1ccc(Br)cc1-n1cccn1', 'Clc1ccc(Br)cc1-n1ccnc1CO', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O[C@H](CO)C(=O)[O-]', 'Clc1ccc(Br)cc1O[C@H](N)c1ccc(F)cc1F', 'Clc1ccc(Br)cc1O[C@H](c1ccccc1)[C@@H]1CCS(=O)(=O)C1', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1S(=O)(c1ccccc1)c1ccccc1', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1N1CCN(C(=O)C(F)(F)F)CC1', 'Clc1ccc(Br)cc1N1/C=Nc2ccc(Cl)cc2C1', 'Clc1ccc(Br)cc1Cc1nnnn1C1CC1', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1/N=N/SC1=NC=N/c2nccnc21', 'Clc1ccc(Br)cc1/N=N\\c1ccc(Br)cc1', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1C(=O)[O-]', 'Clc1ccc(Br)cc1C(F)(F)C1CC1', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1NCC(=O)N1CCC[C@@H](c2[nH+]ccn2CC2CCCC2)C1', 'Clc1ccc(Br)cc1NC(=S)NCc1ncc[nH]1', 'Clc1ccc(Br)cc1NCc1ccc2c(c1)CCC(=O)N2Cc1cc(F)ccc1F', 'Clc1ccc(Br)cc1NC1=CSCCS1', 'Clc1ccc(Br)cc1NC[C@@H](O)c1ccc(Cl)cc1', 'Clc1ccc(Br)cc1NC[C@H](c1ccccc1)[NH+]1CCCC1', 'Clc1ccc(Br)c(F)c1CN(Cc1ccccc1)Cc1ccco1', 'Clc1ccc(Br)c(F)c1Oc1cccc(CO)c1', 'Clc1ccc(Br)c(F)c1NCC1(CC2CC2)CCOCC1', 'Clc1ccc(Br)c(F)c1F', 'Clc1ccc(Br)c(F)c1-c1ccccc1OCc1cc(F)cc(Cl)c1', 'Clc1ccc(Br)c(/C=C2\\N=C(c3ccccc3)N=N2)c1O', 'Clc1ccc(Br)c(/C([O-])=N/N2CC[NH2+]CC2)c1', 'Clc1ccc(-c2nonc2[S-])s1', 'Clc1ccc(-c2nonc2[S-])cc1', 'Clc1ccc(-c2nonc2[S-])nc1', 'Clc1ccc(C[NH2+]CCNc2ccc(Br)cc2)cc1', 'Clc1ccc(C[NH2+]Cc2cc(Br)ccc2O)cc1', 'Clc1ccc(C[NH2+]C2CCSCC2)cc1', 'Clc1ccc(C[NH2+]C[C@@H]2C[C@H]3CC[C@@H]2C3)cc1', 'Clc1ccc(C[NH2+]C[C@H]2CCCc3ccc(F)cc32)o1', 'Clc1ccc(Br)cc1Sc1nc2ccccc2cc1F', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc([C@@H]([NH2+]CC(=O)c2ccc(F)cc2)C2CCCC2)cc1', 'Clc1ccc([C@@H]([NH2+]Cc2ccccc2)C2CCN(C3CCCCC3)CC2)cc1', 'Clc1ccc([C@@H]([NH2+]C2CCN(C[C@@H]3CCOC3)CC2)C2CC2)cc1', 'Clc1ccc([C@@H]([NH2+]C[C@H]2CCCO2)N2Cc3ccccc3C2)nc1Cl', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1-c1cnn(CCc2ccccc2)n1', 'Clc1ccc(Br)cc1-c1cnnn1C(F)(F)F', 'Clc1ccc(Br)cc1-c1cnncn1', 'Clc1ccc(Br)cc1-c1cnn2cc(Br)cnc21', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1S(=O)(c1ccc(F)cc1)C(=O)Nc1ccc(Br)cc1', 'Clc1ccc(Br)cc1S(=O)(=O)Nc1ccc(Cl)cc1', 'Clc1ccc(Br)cc1S(=O)(F)c1cnn(Cc2ccccc2)c1', 'Clc1ccc(Br)cc1S(=O)([O-])Br', 'Clc1ccc(Br)cc1S(=O)(Br)F', 'Clc1ccc(-c2cc[nH]n2)c2ncccc12', 'Clc1ccc(-c2cc[nH]n2)o1\n', 'Clc1ccc(-c2cccnc2Cl)cc1Br', 'Clc1ccc(-c2cc([N+](=O)[O-])c([O-])nn2)cc1', 'Clc1ccc(-c2ccccc2CCNC(=O)COc2ccccc2Cl)cc1', 'Clc1ccc(-c2ccccc2C2(c3ccc(Cl)cc3)CCOCC2)cc1', 'Clc1ccc(-c2ccccc2COC2CCN(c3nc4ccccc4o3)CC2)cc1', 'Clc1ccc(-c2ccccc2CN2CCC[C@](O)(C(F)(F)F)C2)cc1', 'Clc1ccc(-c2cc[nH]n2)nn1', 'Clc1ccc(-c2cc(Cl)ccc2N2CCCCC2)cn1', 'Clc1ccc(-c2cc(NCc3cccs3)no2)cc1', 'Clc1ccc(-c2cc(Nc3ccccc3Cl)nc(-c3ccc(Cl)cc3)n2)cc1', 'Clc1ccc(-c2cc(N3CCOCC3)[nH]c2Br)nc1', 'Clc1ccc(-c2cc(N[C@@H]3CCS(=O)(=O)c4ccccc4C3)ccn2)cc1', 'Clc1ccc(-c2n[nH]c3[nH]cnc23)cc1Cl', 'Clc1ccc(Br)c(OCCSc2ccc(Cl)cc2)c1', 'Clc1ccc(Br)c(OC2CC[NH+](C3CCCC3)CC2)c1', 'Clc1ccc(Br)c(OCOC2CC[NH2+]CC2)c1', 'Clc1ccc(Br)c(OC[C@@H]2C=c3ccccc3=[NH+]2)c1', 'Clc1ccc(Br)c(OC[C@H]2CC[NH+](CC(F)(F)F)C2)n1', 'Clc1ccc(-c2n[nH]cc2CNc2ccc(Cl)c(F)c2F)cc1', 'Clc1ccc(-c2n[nH]cc2-c2ccccc2F)cc1', 'Clc1ccc(-c2n[nH]cc2Br)cc1', 'Clc1ccc(-c2n[nH]cc2CN2CCN(Cc3cccs3)CC2)cc1', 'Clc1ccc(-c2n[nH]cc2NCc2csc3ccccc23)cc1Br', 'Clc1ccc(-c2nn(-c3ccc(COCC(F)F)cc3)o2)cc1', 'Clc1ccc(-c2nn(-c3ccc(Br)cc3)c(=O)n2Cc2cccc(F)c2)cc1', 'Clc1ccc(-c2nn(-c3ccc(F)cc3)cn2)cc1', 'Clc1ccc(-c2nn([C@@H](Cc3ccccc3)c3ccc(F)cc3F)nn2)cc1', 'Clc1ccc(-c2nn([C@@H]3CC[C@@H](O)[C@@H](CO)[C@@H]3O)cc2F)cc1', 'Clc1ccc(-c2nn([C@@H]2CC23CC[NH+](Cc4ccccc4F)CC3)C2)cc1F', 'Clc1ccc([C@@H](NS[C@H]2CCOC3(CCCC3)C2)N2CCOCC2)cc1', 'Clc1ccc([C@@H](NS(=O)(=O)c2ccc(Cl)cc2)N2CCCC2)cc1', 'Clc1ccc([C@@H](NSc2nncc3c2CCC3)C(F)(F)F)cc1', 'Clc1ccc(-c2cc[nH]n2)c([N+](=O)[O-])c1', 'Clc1ccc(-c2cc[nH]n2)c(C[NH2+]CCc2ccccc2)c1', 'Clc1ccc(-c2cc[nH]n2)c(OCCC(=O)[O-])c1', 'Clc1ccc(-c2cc[nH]n2)c(NCc2ccccc2)c1', 'Clc1ccc(-c2cc[nH]n2)c(F)c1', 'Clc1ccc(-c2cc[nH]n2)c(Cl)c1', 'Clc1ccc(-c2cc[nH]n2)c([C@H]2CC[NH2+]C2)n1', 'Clc1ccc(-c2cc[nH]n2)c(Br)c1', 'Clc1ccc(-c2cc[nH]n2)c(/C=C/c2ccccc2)c1', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Cc2nccc(CCCCCl)n2)cc1', 'Clc1ccc(Cc2ncoc2CNc2ccc(-n3cccc3)o2)cc1', 'Clc1ccc(Cc2nc3ccccc3[nH]2)nn1', 'Clc1ccc(COc2ncccc2F)cc1', 'Clc1ccc(COc2nc(CN3CCC(c4ccccc4)CC3)no2)cc1', 'Clc1ccc(COc2ncc(Cl)cc2CO)cc1', 'Clc1ccc(COc2nc3ccccc3c3sccc23)c2ccccc12', 'Clc1ccc(COc2nc[nH]c2Br)cc1', 'Clc1ccc(CNCCCn2ccc[nH+]2)cc1', 'Clc1ccc(CNC(=O)N2CCCCC[C@@H]2c2ccccc2)o1', 'Clc1ccc(CNC2CC[NH+](C[C@H]3CCCO3)CC2)cc1', 'Clc1ccc(CNC[C@@H]2CC[C@@H](C(F)(F)F)O2)cc1[N+](=O)[O-]', 'Clc1ccc(CNC[C@H]2CCCN(C(=O)[C@@H]3CCCO3)C2)cc1', 'Clc1ccc(Br)cc1C[C@@H]1CCCN(C(=O)c2cc([N+](=O)[O-])ccc2F)C1', 'Clc1ccc(Br)cc1O\n\n\n\n\n', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccc(Cl)cc1', 'Clc1ccc(Br)cc1/C=N\\C(=O)C1=NN=C1Cc1ccccc1', 'Clc1ccc(Br)cc1/C=N\\c1ccc(C(C)C)cc1', 'Clc1ccc(Br)cc1/C=N\\OCCN1CCCC1=O', 'Clc1ccc(Br)cc1/C=N\\N=C\\C=C\\c1ccc(Br)cc1', 'Clc1ccc(Br)cc1/C=N\\Sc1ccc(C(=O)N2CCCC2)c2nccnc12', 'Clc1ccc(Br)cc1/C=N\\[C@H]1CCC[C@H](O)C1', 'Clc1ccc(Br)cc1CCCCBr', 'Clc1ccc(Br)cc1CC(=O)N/N=C/c1ccc(F)cc1', 'Clc1ccc(Br)cc1CCc1ccccc1', 'Clc1ccc(Br)cc1CCNc1ccccn1', 'Clc1ccc(Br)cc1CCn1cccn1', 'Clc1ccc(Br)cc1CC[NH+]1CCCC1', 'Clc1ccc(Br)cc1CC/N=c1\\sc(-c2cccc(Cl)c2)n[nH]1', 'Clc1ccc(Br)cc1CC[NH2+]C1(CO)CCCCC1', 'Clc1ccc([C@@H]([O-])CCc2nc(C3CCCCC3)no2)cc1', 'Clc1ccc([C@@H]([O-])Cc2ccc([N+](=O)[O-])cc2)cc1', 'Clc1ccc([C@@H]([O-])C[C@@H](/C=O)c2cc(F)ccc2F)cc1', 'Clc1ccc([C@@H]([O-])Cn2cncn2)c(Cl)c1', 'Clc1ccc([C@@H]([O-])C[NH+](Cc2ccco2)C2CC2)cc1', 'Clc1ccc(Cc2c[nH]nc2[C@H]2CC=CC2)cc1', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n', 'Clc1ccc(CO)nc1O/N=C/c1c(F)cccc1F', 'Clc1ccc(CO)n1CCSc1ccc(F)cc1Cl', 'Clc1ccc(Cc2nnc3scnc3n2)cc1', 'Clc1ccc(Cc2nnn(-c3ccccc3)n2)cc1Cl', 'Clc1ccc(COc2nc3[nH]c4ccccc4n3c2NCCCCCCN2CCOCC2)cc1', 'Clc1ccc(COc2nc3cccc(Cl)c3[n-]2)cc1', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccccc1[N+](=O)[O-]', 'Clc1ccc(Br)cc1-n1nnc2sc(-c3ccncc3)nc21', 'Clc1ccc(Br)cc1-n1nnnc1-c1ccccc1', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n', 'Clc1ccc(-c2csc3[nH]nnc23)cc1O', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccc(F)cc1F', 'Clc1ccc(Br)cc1-c1ccc(Cl)cc1F', 'Clc1ccc(Br)cc1-c1ccc2c(c1)nc(Cn1cccn1)n2C1CC1', 'Clc1ccc(Br)cc1-c1ccccc1N1CCOCC1', 'Clc1ccc(Br)cc1-c1cccnc1', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccc(F)cc1F', 'Clc1ccc(-c2csc3[nH]nnc23)cc1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1Cl', 'Clc1ccc(-c2csc3[nH]c(COCN4CCOCCC4)nc23)nc1', 'Clc1ccc(-c2csc3[nH]c(Nc4cccnc4F)nc23)cc1', 'Clc1ccc(Br)cc1O[C@@H](CCC12CC=CCC2)C1', 'Clc1ccc(Br)cc1O[C@@H](Nc1ccc(F)cc1)n1cccn1', 'Clc1ccc(Br)cc1O[C@@H]([NH3+])C1CCCC1', 'Clc1ccc(Br)cc1O[C@@H](c1ccccc1)N1CCCC1', 'Clc1ccc(C(F)(F)F)cc1OC(=O)c1ccc(Cl)c(Cl)c1', 'Clc1ccc(C(F)(F)F)c(-[n+]2c(C(F)(F)F)nc3ccccn32)c1', 'Clc1ccc(Br)c(SCCn2cncn2)c1', 'Clc1ccc(Br)c(SCc2nnc3n2CCC3)c1', 'Clc1ccc(Br)c(SC[C@@H]2OCCCO2)c1', 'Clc1ccc(Br)c(SC2CCCC2)n1', 'Clc1ccc(Br)c(F)c1-c1cc([N+](=O)[O-])cnc1-c1ccccc1', 'Clc1ccc(Br)c(F)c1-n1cncn1', 'Clc1ccc(Br)c(N2CCCCCC2)c1', 'Clc1ccc(CO)c(N2C(=O)CNC2=O)c1', 'Clc1ccc(CO)cc1OCCn1nccc1CN1CCN(Cc2ccccc2)CC1', 'Clc1ccc(COc2nc([N+](=O)[O-])ccc2N2CCCCC2)cc1', 'Clc1ccc(COc2nc(C3CC3)n(-c3ccccc3)n2)cc1', 'Clc1ccc(COc2nc(N3CCCCC3)[nH]c2N2CC[C@H](C[NH+]3CCCC3)C2)cc1', 'Clc1ccc(COc2nc(F)cc(Br)c2)c(F)c1', 'Clc1ccc(COc2nc(-c3cccc(Br)c3)co2)cc1', 'Clc1ccc(COc2nc(Cl)cc(Cl)c2Cl)cc1', 'Clc1ccc(COc2nc([O-])c(C(=O)[O-])cc2Cl)cc1', 'Clc1ccc(COc2nc(Br)ccn2)cc1', 'Clc1ccc(C(C)C)cc1CN/C(=C/c1ccccc1)c1ccccc1', 'Clc1ccc(C(C)(C)CN2CCNC[C@H]2c2ccccc2)cc1', 'Clc1ccc(C(=O)NCCOc2ccccc2)cc1', 'Clc1ccc(C(=O)N2CCCNC2)cc1F', 'Clc1ccc(Cc2noc(-c3ccc(O)cc3)n2)cc1', 'Clc1ccc(C(F)(F)F)c(F)c1Cl', 'Clc1ccc(CC[C@H]([NH3+])[C@H]2COc3ccccc3C2)o1', 'Clc1ccc(CC[C@H]2C[C@H]3CC[C@@H]2CN(C(=O)NCCC(=O)N4CCCC4)C3)cc1', 'Clc1ccc(-c2nonc2)nc1', 'Clc1ccc(-c2nonc2)nn1', 'Clc1ccc(Br)cc1-n1nnnc1NC[C@@H]1CCCCO1', 'Clc1ccc(-c2cccnc2)nc1C[C@@H](O)COc1ccccc1Br', 'Clc1ccc(-c2cccnc2S(=O)(=O)C(F)(F)F)cc1', 'Clc1ccc(-c2cccnc2SC[C@@H]2CCCO2)cc1', 'Clc1ccc(-c2cccnc2Sc2nnc3ccccn23)cc1', 'Clc1ccc(-c2cccs2)nc1C[C@H]1CCCCN1', 'Clc1ccc(-c2cccs2)nn1', 'Clc1ccc(-c2ccccc2NC2CCCCC2)cc1', 'Clc1ccc(-c2ccccc2N2CCO[C@@H](CO)C2)cc1', 'Clc1ccc(-c2cccs2)[nH]c1=O', 'Clc1ccc(-c2cccs2)[nH]1', 'Clc1ccc(-c2cc(/C=C/c3cccnc3)n[nH]2)cc1', 'Clc1ccc(-c2cc(/[NH+]=C/c3ccco3)nc([S@](=O)c3ccccc3)n2)cc1', 'Clc1ccc(-c2nonc2N2CCC[NH2+]CC2)cc1Cl', 'Clc1ccc(Cc2noc(C3CC3)n2)cc1', 'Clc1ccc(Cc2noc(Cc3cccs3)n2)cn1', 'Clc1ccc(Cc2noc(-c3ccco3)c2Cl)cc1', 'Clc1ccc(Cc2noc([C@H]3COc4ccccc4O3)n2)cc1', 'Clc1ccc(Cc2noc(/N=C/C=C/c3ccccc3)n2)cc1Cl', 'Clc1ccc(C(CCCO)C(F)F)cc1', 'Clc1ccc(C(CC)(NCC(F)(F)F)C(F)(F)F)cc1', 'Clc1ccc(C(CCF)N2CCC[C@H](c3nc4ccccc4[nH]3)C2)cc1', 'Clc1ccc(C(CCn2cccn2)C(F)(F)F)cc1', 'Clc1ccc(C(CCBr)COCCO)s1', 'Clc1ccc(C2SCCS2)cc1', 'Clc1ccc(C2=CC[NH+](Cc3ccccc3)CC2)cc1', 'Clc1ccc([C@@H]([C@@H]O)N[C@@H]2CCC[C@@H](O)C2)cc1', 'Clc1ccc(Cc2c[nH]nc2-c2ccccc2)cc1', 'Clc1ccc(C2SCCS2)cc1', 'Clc1ccc(CO)cc1OCc1cscn1', 'Clc1ccc(Br)cc1N1CCN(CC2[NH+]=c3ccccc3=[NH+]2)CC1', 'Clc1ccc(Br)cc1N1C(=O)/C(=C\\c2cccnc2)SC1=S', 'Clc1ccc(Br)cc1N1C[C@H](c2ccccc2)CC1', 'Clc1ccc(Br)c(Sc2nc(=O)n3ccsc3n2)c1', 'Clc1ccc(Br)cc1[C@@H](Cl)S(=O)(=O)c1ccccc1', 'Clc1ccc(Br)cc1-c1ccn(C[C@H]2CCCO2)n1', 'Clc1ccc(Br)cc1-c1ccncc1', 'Clc1ccc(-c2ccccc2)cc1Nc1ccc2nnc(-c3ccccc3)n2c1', 'Clc1ccc(-c2ccccc2)nc1CN[C@H]1CCCC[C@H]1CO', 'Clc1ccc(-c2cccnc2N=C2CCCCC2)cc1', 'Clc1ccc(-c2cccnc2N2CCCSC2)cc1', 'Clc1ccc(-c2ccccc2Cl)nc1O', 'Clc1ccc(-c2ccccc2[N-]CCO)cc1', 'Clc1ccc(-c2ccccc2[N-][C@@H]2CN[C@@H](c3ccc(Cl)cc3)C2)cn1', 'Clc1ccc(-c2csc3scnc23)cc1', 'Clc1ccc(-c2csc3sccc23)cc1F', 'Clc1ccc(-c2csc3scnc32)cc1', 'Clc1ccc(-c2csc3scnc23)c[nH+]1', 'Clc1ccc([C@@H](Nc2nccs2)C2CC2)cc1', 'Clc1ccc([C@@H](Cl)CC[C@@H]2CCCCN(S(=O)(=O)c3ccccc3)C2)cc1', 'Clc1ccc([C@@H](Cl)C(=O)Nc2ccc(F)c(F)c2)cc1', 'Clc1ccc([C@@H](Cl)Cc2nccn2Cc2ccccc2)cc1', 'Clc1ccc([C@@H](Cl)C2CC2)c2ncccc12', 'Clc1ccc([C@@H](Cl)C[C@@H]2COc3ccccc3C2)cc1', 'Clc1ccc([C@@H](Cl)C[C@H](Cc2ccccc2)N2CCOCC2)cc1', 'Clc1ccc(C2SCCOC2)nc1', 'Clc1ccc(C2SCCCC2)cc1', 'Clc1ccc(C2SCCS2)cc1O', 'Clc1ccc(Br)c(/[NH+]=C/c2ccccc2)c1', 'Clc1ccc(Br)c([C@@H]2OCCc3ccc(F)cc32)c1', 'Clc1ccc(Br)c([C@@H]2C[C@@H]2c2ccccc2)c1', 'Clc1ccc(Br)c([C@@H]2SCCC[C@@H]2N2CCOCC2)c1', 'Clc1ccc(Br)c([C@@H]2[C@H](C(=O)[O-])CCO2)n1', 'Clc1ccccc1-c1cncnc1NC[C@@H]1C[C@H]1c1ccccc1', 'Clc1ccccc1-c1cncc(Br)c1', 'Clc1ccccc1-c1cnc2n1CCC[C@@H]2N[C@@H]1CCS(=O)(=O)CC1', 'Clc1ccccc1OC1(C[C@@H]2CCCCCC2)CC1', 'Clc1ccccc1OC1(OCc2ccccc2)CCCC1', 'Clc1ccccc1OC1(NC(=O)C(=O)c2ccccc2)CC1', 'Clc1ccccc1OC1(F)CN(Cc2ccncc2)CC1', 'Clc1ccccc1OC1([C@@H](O)c2ccc(F)cc2)CC1', 'Clc1ccccc1OC1(Cl)CCN(C(=O)c2ccccc2I)CC1', 'Clc1ccccc1OC1([NH+]2CCCCC2)Cc2ccccc2C1', 'Clc1ccccc1OC1(Br)CC1', 'Clc1ccc(-c2nnnn2-c2ccccc2)c2c1CCC2', 'Clc1ccc(-c2nnnn2Cl)cc1', 'Clc1ccc(-c2nnnn2-c2cccc(Cl)c2)cc1', 'Clc1ccc(-c2nnnn2Cl)cc1[N+](=O)[O-]', 'Clc1ccc(-c2nnnn2Cl)cn1', 'Clc1ccc(-c2n[nH]cc2-c2ccccc2F)cc1', 'Clc1ccc(-c2n[nH]cc2-n2ccc([N+](=O)[O-])c2)cc1', 'Clc1ccc(-c2n[nH]c(C(=O)N3CCCC3)c2)cc1', 'Clc1ccc(-c2n[nH]c(CO[C@@H]3CC[C@@H]4CCCC[C@@H]4C3)c2[N+](=O)[O-])cc1Cl', 'Clc1ccc(-c2n[nH]c(CN(Cc3ccco3)C2CCCCCCCOCCOCCO)c2)ccn12', 'Clc1ccc(-c2n[nH]c(C[NH+]3CCC(COc4ccc(F)c(F)c4)CC3)n2)cc1', 'Clc1ccc(-c2n[nH]c(C[NH2+]c3ccc(N4CCOCC4)cc3)n2)cc1', 'Clc1ccc(COCOC2CCCC2)cc1', 'Clc1ccc(COCO[C@H]2CCC[NH2+]C2)cc1', 'Clc1ccc(COCOc2ccccc2F)cc1', 'Clc1ccc(COCO[C@@H]2CCO[C@]2(F)c2ccc(F)cc2)cc1', 'Clc1ccc([C@@H](NNC(=O)c2ccc(N3CCCCC3=O)cc2)N2CCCCC2)cc1', 'Clc1ccc([C@@H](NNc2nc3ccccc3s2)C2CC2)cc1', 'Clc1ccc([C@@H](NN2CCOCC2)c2cc(F)ccc2F)cc1', 'Clc1ccc([C@@H](NN[C@@H]2CCc3ccccc32)[NH+]2CCCC2)cc1', 'Clc1ccc([C@@H](NN/c2ccc(Cl)cc2Cl)c2cc(F)ccc2F)cc1', 'Clc1ccc(Br)cc1[C@@H]1CCCN(C(=O)Nc2ccc3c(c2)OCO3)C1', 'Clc1ccc(Br)cc1[C@@H]1Cc2ccccc2S1', 'Clc1ccc(Br)cc1[C@@H]1COCC[NH+](Cc2ccco2)C1', 'Clc1ccc(Br)cc1[C@@H]1C[C@@H](CI)c2[nH]cnc2S1', 'Clc1ccc(Br)cc1[C@@H]1CSCN1c1ccc(F)cc1', 'Clc1ccc(Br)cc1[C@@H]1C[C@H]1c1ccccc1F', 'Clc1ccc(Br)cc1[C@@H]1C[NH+](C2CC2)CN1Cc1cccc(F)c1', 'Clc1ccc(Br)cc1[C@@H]1C[NH2+][C@@H](CO)[C@H]2NC(=S)N(Cc3ccccc3)C12', 'Clc1cccc(F)c1OCC(=O)NCc1ccc(C(=O)[O-])cc1', 'Clc1cccc(F)c1Oc1ccccc1', 'Clc1cccc(F)c1O[C@@H]1CCO[C@]2(CCSC2)C1', 'Clc1cccc(F)c1O[C@H]1C[C@H]2CC[C@@H]1C2', 'Clc1cccc(F)c1O/N=C/c1cc(Cl)c(Cl)c(Br)c1', 'Clc1ccc(C(F)(F)F)nc1N1CCCCC1', 'Clc1ccc(C(=O)Oc2ccccc2C(=O)[O-])cc1', 'Clc1ccc(C(=O)N2CCN(Cc3c[nH]nc3C3CC3)CC2)c(F)c1', 'Clc1ccc(C(=O)c2ccc(O)cc2F)cc1', 'Clc1ccc(CN[C@@H](c2ccccc2)C2CC2)cc1', 'Clc1ccc(CN[C@@H]2CCCC[C@H]2[S@@](=O)Cn2cncn2)cc1', 'Clc1ccc(C(F)(F)F)c(F)c1', 'Clc1ccc(C(F)(F)F)nc1OCCCOc1ccccc1', 'Clc1ccc(C[NH+](Cc2ccccc2)CN2CCN(c3ccccc3C(F)(F)F)CC2)cc1', 'Clc1ccc(C[NH+](CCO)Cc2cccnc2)cc1', 'Clc1ccc(C[NH+](Cc2ccc(F)cc2)[C@H]2CCOC2)cn1', 'Clc1ccc(C[NH+](C[C@@H]2Cc3ccccc3O2)c2nccn2Cc2ccccc2)cc1', 'Clc1ccc([C@@H]([C@@H](Cl)c2ncc[nH]2)N2CCOCC2)cc1', 'Clc1ccc([C@@H]([C@@H](Br)c2ccccc2)C(F)(F)Cl)cc1', 'Clc1ccc([C@@H]([C@@H](O)[C@H]2CCCO2)Sc2ccccc2)nc1', 'Clc1ccc(-c2cc([C@H](NC3CC3)C(=O)N3CCCC3)ccc2Cl)cc1', 'Clc1ccc(-c2cc([C@H]3CCCO3)n[nH]2)cc1', 'Clc1ccc(C(N)=[NH2+])cc1', 'Clc1ccc(C2=CC[C@@H]2C2CCC2)cc1F', 'Clc1ccc(C2=Cc3ccc([N+](=O)[O-])cc3O2)cn1', 'Clc1ccc(C2=C[C@@H](O)[C@]3(CCCN(c4ccccc4)C3)C2)cc1', 'Clc1ccc(C2=CSCCN2)cc1[N+](=O)[O-]', 'Clc1ccc(C2=C[C@H](S(=O)(=O)Cc3ccc(Br)cc3F)N2[N+](=O)[O-])cc1', 'Clc1ccc(-c2cccs2)nn1', 'Clc1ccc(-c2cc(F)ccc2F)cc1', 'Clc1ccc(-c2cc(Br)oc2C2CC2)o1', 'Clc1ccc(-c2cc(Br)ccc2N2CCCC2)cc1', 'Clc1ccc(-c2cc(Br)nc(N3CCC(CN4CCCC4)CC3)n2)cc1', 'Clc1ccc(-c2cccnc2Cl)cc1', 'Clc1ccc(-c2cccnc2Cl)nc1', 'Clc1ccc(-c2cccs2)[nH]c1=O', 'Clc1ccc(-c2cccnc2)n2nnc(-c3ccc([N+](=O)[O-])cc3)c12', 'Clc1ccc(-c2cccnc2)nc1O[C@@H]1CCOC2(CCCCC2)C1', 'Clc1ccc(-c2cccnc2)nn1', 'Clc1ccc(-c2cc(=O)oc(=O)c3ccccc23)cc1', 'Clc1ccc([C@@H]([NH2+][C@@H](CO)c2ccccc2)C2CC2)cc1', 'Clc1ccc([C@@H]([NH2+][C@@H]2CCOC2)c2ccc(Cl)cc2)cc1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1', 'Clc1ccc(-c2csc3[nH]nnc23)s1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1', 'Clc1ccc(Br)cc1-n1ncc(Br)c1C(=O)[O-]', 'Clc1ccc(Br)cc1-n1nccc1CSc1nnnn1C1CC1', 'Clc1ccc(Br)cc1-n1ncc2c1[C@H]2CCCOc1ccccc1', 'Clc1ccc(-c2cc[nH]n2)o1\n\n', 'Clc1ccc([C@@H](Br)[C@H](Cc2ccccc2)[NH+](C)C2CC2)cc1', 'Clc1ccc([C@@H](Br)[C@H]2CC[C@@H]3CCC[C@H]23)cc1', 'Clc1cccc(S(=O)(=O)Oc2ccccc2Br)c1', 'Clc1cccc(SCC/C=C/CO)n1', 'Clc1cccc(SCc2nnnn2Cc2ccccc2)c1', 'Clc1cccc(SC2CCC2)c1', 'Clc1cccc(SC[C@@H](O)c2ccc(Cl)cc2F)c1', 'Clc1cccc(SC[C@H]2CCCC[C@@H]2[NH2+][C@@H]2CCOC2)c1', 'Clc1cccc(Br)c1C[N+]1=CCCC1', 'Clc1cccc(Br)c1CC1OCCC[C@@H]1C(=O)[O-]', 'Clc1cccc(Br)c1C(F)(F)F', 'Clc1cccc(Br)c1COCc1ccc(Cl)cc1', 'Clc1cccc(Br)c1CNc1cc([N+](=O)[O-])ccc1O', 'Clc1cccc(Br)c1Cn1nnc2cc(F)ccc21', 'Clc1cccc(Br)c1CSCCCCC[NH+]1CCCCC1', 'Clc1cccc(Br)c1C[C@H](O)c1cnccn1', 'Clc1cccc(Br)c1C[NH+]1CCCC[C@@H]1CCl', 'Clc1cccc(Br)c1C[NH2+][C@H]1CCC[C@H](Cc2ccccc2)C1', 'Clc1ccc(C[NH2+]c2cccnc2)cc1', 'Clc1ccc(-c2cccs2)nc1CN1CCCc2ccccc21', 'Clc1ccc(-c2cccs2)[nH]1', 'Clc1ccc(-c2cc[nH]n2)nc1', 'Clc1ccc(-c2cc[nH]n2)nc1C[NH2+]CC[C@H]1OCCO1', 'Clc1ccc(-c2cc[nH]n2)nc1O', 'Clc1ccc(-c2cc[nH]n2)nc1NC[C@H](O)COc1cccnc1', 'Clc1ccc(-c2cc[nH]n2)nc1SC[C@@H]1CN(C2CC2)CCO1', 'Clc1ccc(-c2cc[nH]n2)nc1[C@H]1COCCO1', 'Clc1ccccc1CI\n\n', 'Clc1ccccc1C(=O)[O-]', 'Clc1ccccc1CI\n\n\n', 'Clc1ccccc1C(CCCO)C[NH2+]C[C@@H]1CCCO1', 'Clc1ccccc1C(C(F)(F)F)[C@@H]1CCN(c2ccccc2)C1=O', 'Clc1ccccc1C(C)CCOc1c(Cl)cccc1Cl', 'Clc1ccccc1C(C1CC1)[C@H]1CCC[C@@](O)(Cn2cccn2)C1', 'Clc1ccccc1C(CO)(Cl)[C@H](O)c1ccccc1', 'Clc1ccccc1Oc1cc(Cl)c(Cl)cc1Cl', 'Clc1ccccc1Oc1cc(F)c(F)c(F)c1F', 'Clc1ccccc1Oc1cc(OCc2ccccc2)nc(-c2ccccc2)n1', 'Clc1ccccc1Oc1cc(O)ccc1F', 'Clc1ccccc1Oc1cc(Oc2ccccc2)nnc1-c1ccccc1', 'Clc1ccccc1Oc1cc(O[C@@H]2CCOC3(CCOCC3)C2)nc2ccccc12', 'Clc1ccccc1O/C=CCN1C(=O)CN(c2ccc(Cl)cc2)C1=O', 'Clc1ccccc1O/C=Cc1ccc(N2S(=O)([O-])N=C(O)COc3ccccc32)cc1', 'Clc1ccccc1O/C=C1/C(=N/CC2CCCCC2)N=N1', 'Clc1ccccc1O/C=COc1ccc([N+](=O)[O-])cc1', 'Clc1ccccc1O/C=C[C@@H](O)COCc1ccccc1', 'Clc1ccccc1O/C=C/CN1CCCSCC1', 'Clc1ccc(CO)n1-c1ccccc1', 'Clc1ccc(CO)n1CC12CC3CC(CC(C3)C1)C2', 'Clc1ccc(CO)n1[C@@H]1CC(=O)N(Cc2ccco2)C1=O', 'Clc1ccc(Br)c(-n2nnc3cc([N+](=O)[O-])ccc32)c1', 'Clc1ccc(Br)c(Cl)c1\n', 'Clc1ccc(Br)c(-n2nc(COC(=O)C3CCCC3)cc2Cl)c1', 'Clc1ccc(Br)c(-n2nnnc2Sc2ccccc2)n1', 'Clc1ccc(Br)c(F)c1CCC(=O)NCCc1ncn[nH]1', 'Clc1ccc(Br)c(F)c1C(=O)NC[C@@H]1CCCc2ccccc21', 'Clc1ccc(Br)c(F)c1COC[C@@H]1Cc2ccccc2S1', 'Clc1ccc(Br)c(F)c1CN[C@@H](c1nccs1)c1ccccn1', 'Clc1ccc(Br)c(F)c1Cn1c[nH+]c(-c2cccc(Cl)c2)n1', 'Clc1ccc(Br)c(F)c1C[NH+]1CCC[C@H](c2nncn2C2CC2)C1', 'Clc1ccc(Br)c(F)c1C[NH2+][C@@H]1CC[C@H](CCc2ccccc2)C1', 'Clc1cccc2[nH]nnc1c1ccccc12', 'Clc1cc(Cl)ccc1CN1CCCCC[C@@H]1c1ccccn1', 'Clc1cc(Cl)ccc1OS(=O)(=O)c1ccc2c(c1)CCCC2', 'Clc1cc(Cl)ccc1I', 'Clc1cc(Cl)ccc1-c1ccccc1CCNC(=O)N1CCOCC1', 'Clc1cc(Cl)ccc1Cl', 'Clc1cc(/N=C/c2c(Cl)cccc2Cl)nc2ccccc12', 'Clc1cc(/N=C/c2ccccc2)c(Br)o1', 'Clc1cc(/N=C/c2cnc(-c3cccs3)s2)ccc1NC[C@@H](O)c1ccccc1', 'Clc1cc(Cl)c2sc(N[C@H]3CCS(=O)(=O)C3)nc2c1', 'Clc1cc(Cl)c2sccc2c1Oc1ccccc1', 'Clc1cc(Cl)c2scnc2n1', 'Clc1cc(/N=C/c2noc(-c3ccccc3)n2)cc(Cl)c1', 'Clc1cc(/N=C/c2n[nH]n3cccc23)ccc1O', 'Clc1cc(/N=C/c2nc3ccccc3[nH]2)c([N+](=O)[O-])cc1O', 'Clc1cc(/N=C/c2nnc3ccccn23)cc(OCc2ccccc2)c1Cl', 'Clc1cc(NC[C@@](O)(c2cccc(F)c2)n2nc(C(F)(F)F)cc2)ccc1Cl', 'Clc1cc(NC[C@@]2(C(=O)[O-])CCCN2S(=O)(=O)c2ccccc2)sc1C1CC1', 'Clc1ccc([C@@H](CCCCl)NC(=O)c2ccccc2)cc1', 'Clc1ccc([C@@H](CC(=O)N2CCOc3ccc(Cl)cc32)N2CCOCC2)cc1', 'Clc1ccc([C@@H](CCc2nc([C@H]3CCCN3c3ncc[nH]3)no2)c2ccccc2)cc1', 'Clc1ccc([C@@H](CC2COc3ccccc3CC2)NCCc2cccnc2)s1', 'Clc1ccc([C@@H](CCN2CCOCC2)[C@@H]2CCS(=O)(=O)C2)s1', 'Clc1ccc([C@@H](CC[C@H]2CCOC2)NN=C2CCCOC2)cc1', 'Clc1cc(F)cc2nc(CC3CCN(c4ccccc4F)CC3)nn12', 'Clc1cc(F)cc2nnc(N[C@@H]3CCCOC3)n12', 'Clc1cc(Cl)c2c(c1)N(Cc1ccc(Cl)cc1)CCCC2', 'Clc1cc(Cl)c2c(n1)CCC[C@@H]2[NH2+][C@@H](c1ccccc1)c1cccc(N2CCOCC2)c1', 'Clc1ccc(Br)cc1-c1cn(-c2ccccc2)nc1-c1ccccc1', 'Clc1cccc(/C(CC(=O)Nc2cc3c(cc2Cl)OCCO3)[NH+]2CCCC2)c1', 'Clc1cccc(/C(c2ccccn2)N2CCN(CC(=O)Nc3ccc(Cl)cc3)CC2)c1', 'Clc1cccc(/C(=C/Nc2ccc(F)c(F)c2)C(=O)[O-])c1', 'Clc1cccc(/C(Cl)/N=C/C=C\\c2ccco2)c1', 'Clc1cccc(/C([O-])/N=C(\\[O-])CC[C@@H](C(=O)[O-])c2ccccc2)c1', 'Clc1cccc(/C(Br)=C/CN2CCOCC2)c1', 'Clc1ccc(C(F)(F)F)nc1OCCCCCO', 'Clc1ccc(C(F)(F)F)n1NCC1CCN(C(=O)CC2=COCCO2)CC1', 'Clc1ccc(Cc2nnn(CC(=O)NCCN3CCOCC3)c2Cl)cn1', 'Clc1ccc(Cc2nnn[n-]2)cc1', 'Clc1ccc(Cc2nnnn2-c2ccccc2F)cc1', 'Clc1ccc(C(N)=NC2=NS(=O)(=O)c3ccccc32)cc1', 'Clc1ccc(C(N)=N\\O[C@H]2CCO[C@]3(CCOC3)C2)cc1', 'Clc1ccc(C(N)=Nc2ccc([N+](=O)[O-])cc2)cc1', 'Clc1ccc(C(N)=NSc2ccc(F)cc2)c(Br)c1', 'Clc1ccc(C(N)=N/S(=O)(=O)CCCC(=O)OC(F)(F)F)cc1', 'Clc1ccc(-c2nnnn2-c2cccc(F)c2F)cc1', 'Clc1ccc(-c2nnnn2/N=C/c2ccccc2Cl)cc1', 'Clc1ccc(-c2nnnn2-c2cc([N+](=O)[O-])ccc2F)cc1', 'Clc1ccc(-c2nnnn2-c2ccccc2)cc1', 'Clc1ccc(-c2nnnn2/N=C/c2cccc([N+](=O)[O-])c2)cc1', 'Clc1ccc(-c2nnnn2[C@@H](Cc2ccccc2)c2noc(C3CC3)n2)cc1', 'Clc1ccc(-c2nnnn2[C@@H](O)c2ccccc2Br)cc1', 'Clc1ccc(-c2nnnn2[C@@H](c2ccccc2)c2ccccc2F)cc1', 'Clc1ccc(-c2nnnn2[C@@H]([NH2+]CCCN2CCCC2)C2CC2)cc1', 'Clc1ccc(-c2nonc2)c(C(=O)[O-])c1', 'Clc1ccc(-c2nonc2)cc1', 'Clc1ccc(-c2nonc2)cn1', 'Clc1ccc(Br)cc1-c1cc2c(cc1Cl)NC[C@@H](c1ccc(F)cc1)N2', 'Clc1ccc(Br)cc1-c1cc2ncccc2nc1Cl', 'Clc1ccc(Br)cc1/N=N\\C(=C\\c1nc2c(Cl)cccc2s1)C(=O)[O-]', 'Clc1ccc(Br)cc1/N=N\\c1ccc(Cl)cc1[N+](=O)[O-]', 'Clc1ccc(Br)cc1/N=N\\OCc1ccc(Cl)cc1Cl', 'Clc1ccc(Br)cc1/N=N\\Nc1cccc(C(=O)[O-])c1', 'Clc1ccc(Br)cc1/N=N\\[C@@H]1CCCC[C@@H]1S(C)(=O)=O', 'Clc1ccc(Br)cc1/N=N\\n1ccc2ccccc21', 'Clc1ccc(Br)cc1/N=N\\S(=O)(=O)CCN1CCCC1=O', 'Clc1ccccc1NS[C@H](Nc1cccnc1Cl)c1ccccc1', 'Clc1ccccc1NS[C@H](C(=O)[O-])C(=O)Nc1ccccc1', 'Clc1ccccc1NS[C@H](O)c1ccc(Br)cc1', 'Clc1ccccc1NS[C@H](c1ccccc1)N1CCOCC1', 'Clc1ccc(C[NH2+][C@H](c2ccccc2)c2cccnc2)cc1', 'Clc1ccc(C[NH2+][C@H]2CCC[C@H](n3ccnn3)C2)cc1', 'Clc1ccccc1-c1nn(-c2ccc(Br)cc2)cc1[O-]', 'Clc1ccccc1-c1nn(Cc2ccnc(COC(F)(F)F)c2)nc1-c1ccc(Br)cc1', 'Clc1ccccc1-c1nc(CN2CCOCC2)nc2nc[nH]c12', 'Clc1ccccc1-c1nc(N2CCCCCCC2)co1', 'Clc1ccccc1-c1nc(-c2ccccc2)sc1C(=O)N/N=C/c1ccccc1', 'Clc1ccsc1Nc1nc2ccc(F)cc2s1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1/CCCn1ccc(=O)c2ccccc21', 'Clc1ccsc1/CC(=O)N/N=C/c1ccc2cc(Cl)ccc2n1', 'Clc1ccsc1/CCc1cnc(-c2cccs2)n1C1(C#N)CCCC1', 'Clc1ccsc1/CC1(C#N)CCN([C@@H]2[C@H](CN3CCN(c4ccccc4)CC3)C2)C1', 'Clc1ccsc1/CCN(C(=O)c1ccc(N(C2CC2)c2ccccc2)cc1)C1CC1', 'Clc1ccsc1N[C@H](C1CCOCC1)S(=O)(=O)c1cccc2nonc12', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1S(=O)(=O)C1CC[NH2+]CC1', 'Clc1ccsc1S(=O)(=O)[N-]c1cccc(C(=O)NOc2ccccc2)c1', 'Clc1ccsc1S(=O)(=O)c1cc(Cl)cc(Cl)c1', 'Clc1ccsc1S(=O)(=O)OC1CCC(c2nccn2Cc2ccccc2)CC1', 'Clc1ccsc1S(=O)(=O)N1CCC(N(Cc2ccncc2)C(=O)c2ccc(F)cc2)CC1', 'Clc1ccsc1S(=O)(=O)[C@@H]1CCCO1', 'Clc1cc(/C=C(\\C#N)C(=O)Nc2ccc(I)cc2)ccc1Cl', 'Clc1cc(/C=C(\\Nc2ccccc2)c2ccccc2)ccc1Cl', 'Clc1cc(/C=C(\\F)C(F)(F)F)ccc1Cl', 'Clc1cc(/C=C(\\Cl)CO)ccc1OCOc1cccc(Cl)c1', 'Clc1cc(/C=C2\\C(=O)N=C(Nc3nc(-c4ccccc4)cs3)C[C@@H]2c2ccc(F)cc2)n[nH]1', 'Clc1cc(/C=C2\\c3ccccc3C(=O)N2c2cccs2)ccn1', 'Clc1cc(/C=C2\\SC(=S)N(C3CC3)C2=O)cs1', 'Clc1cc(/C=N/n2cccc2)nc(C(=O)[O-])c1', 'Clc1cc(/N=C\\c2csc(-c3ccc(F)cc3)n2)ccc1Br', 'Clc1cc(/N=C\\c2c([O-])[nH]c3ccccc32)[nH]c(=O)c1Br', 'Clc1cc(/N=C\\c2c[nH]c[nH+]2)cs1', 'Clc1cc(/N=C\\c2ccc(F)cc2F)c2[nH]cnc2n1', 'Clc1cc(F)c(/C=N/NC(=O)c2ccco2)s1', 'Clc1cc(F)c(/N=C\\c2ccc(Cl)cc2C)cc1O', 'Clc1cc(F)c(C(=O)N2CCc3nnccc32)cc1F', 'Clc1cc(F)c(C2C[C@@H](CO)CC2)nc1S(=O)(=O)N1CCCCC1', 'Clc1cc(F)c(CN2CCC[C@@H](Cc3cccc4cccnc34)C2)cc1', 'Clc1cc(F)c(C[C@@H]2CCC[C@@H]2O)cc1', 'Clc1cc(F)c(C[NH+]2CCCC[C@@H]2CO)cc1F', 'Clc1ccc(-c2csc3sccc3c2=O)cc1', 'Clc1ccc(-c2csc3sccc23)cc1', 'Clc1ccc(-c2csc3s/cnc23)cc1O', 'Clc1ccc(-c2nonc2[S-])cc1', 'Clc1ccc(Br)cc1N1/C=C/Nc2ccccc21', 'Clc1ccc(Br)cc1N1/N=C/c2ccccc2N1CCCc1ccccc1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'Clc1ccc(-c2csc3s/cnc23)cc1Cl', 'Clc1ccc(Br)cc1-c1cn(-c2ccccc2F)nc1-c1ccccc1', 'Clc1ccc(CN2CCc3ccccc3C2)c(F)c1', 'Clc1ccc(CN2C(=O)NC3(CCN(CC(=O)N4CCOCC4)CC3)C2=O)cc1', 'Clc1ccc(CN2C[C@H](Cc3ccccc3)O[C@H](c3ccccc3)C2)cc1Cl', 'Clc1ccc(CN2C[C@@H](c3n[nH]c(=O)[nH]3)CC2)c2ccccc12', 'Clc1ccc(CNS(=O)(=O)c2c(Cl)cccc2Cl)cc1', 'Clc1cccc(C2=NOc3cc4ccccc4oc3=[NH+]2)c1', 'Clc1cccc(C2SCCCOC2)c1Oc1ccccc1', 'Clc1cccc(C2(CO)CC2)c1Cl', 'Clc1cc(C[NH+](Cc2ccco2)Cc2ccc(Cl)o2)co1', 'Clc1ccc(Br)c(S(=O)(=O)N2CCSCC2)c1', 'Clc1ccc(Br)c(Cl)c1Cl', 'Clc1ccc(Br)c(/N=[N+](\\[O-])c2ccc([N+](=O)[O-])cc2F)c1', 'Clc1ccc(Br)c(/N=C/C=C/C=C2\\C(=O)N(Cc3ccccc3)S/C2=N\\c2ccco2)c1', 'Clc1ccc(Br)c(/N=N/Sc2nnc(N3CCOCC3)s2)c1', 'Clc1ccc(Br)cc1-c1cn(-c2ccccc2)nc1-c1cccc2nccnc12', 'Clc1ccc(Br)cc1-c1cn(-c2ccccc2)nn1', 'Clc1ccc(Br)cc1OCCCOCC(F)(F)F', 'Clc1ccc(Br)cc1OCC(=O)NC(C1CC1)C1CC1', 'Clc1ccc(Br)cc1OCC1CCCCC1', 'Clc1ccc(Br)cc1OCCOCc1ccccc1', 'Clc1ccc(Br)cc1OCCNC(=O)c1ccc(=O)n(CCC(F)F)c1', 'Clc1ccc(Br)cc1OCC[C@@H]1CNC(=O)N1Cc1ccccc1', 'Clc1ccc(Br)cc1OCCn1ccccc1=O', 'Clc1ccc(Br)cc1OCCS(=O)(=O)c1ccc(F)c(F)c1F', 'Clc1ccc(Br)cc1OCC[C@H]1COc2ccccc2O1', 'Clc1ccc(Br)cc1-n1cnc(Cc2ccccc2)n1', 'Clc1ccc(Br)cc1-n1cnc2ccccc21', 'Clc1ccc(Br)cc1-n1cncc1Cn1ccnn1', 'Clc1ccc(Br)cc1-n1cncn1', 'Clc1ccc(Br)cc1C[NH+](Cc1cc(O)ccc1C)[C@H]1CCSC1', 'Clc1ccc(Br)cc1C[NH+]1CCC[C@H]1COc1ccc(Cl)cc1', 'Clc1ccc(Br)c(S(=O)(=O)N2CCC(OCc3ccccc3)CC2)c1', 'Clc1ccc(Br)c(Cl)c1CCOCC1CCN(C(=O)[C@@H]2CC=CCC2)CC1', 'Clc1ccc(Br)c(Cl)c1C(=O)N1CCc2ncc(-c3cccc(Cl)c3)nc2C1', 'Clc1ccc(Br)c(Cl)c1C1CCN(Cc2cn3cc(-c4ccccc4)cc3nc2)CC1', 'Clc1ccc(Br)c(Cl)c1COCc1ccccc1Br', 'Clc1ccc(Br)c(Cl)c1CN1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1', 'Clc1ccc(Br)c(Cl)c1C[NH+]1CCC(O)CC1', 'Clc1ccc(Br)c(Cl)c1C[NH2+]Cc1cccc([N+](=O)[O-])c1', 'Clc1ccc([C@@H](O)c2ccc3c(c2)CCO3)cc1Cl', 'Clc1ccc([C@@H](N[C@@H](CO)c2ccc(Cl)cc2)[NH+]2CCCC2)c(F)c1', 'Clc1ccc([C@@H](N[C@@H](O)CO)C(F)(F)F)cc1', 'Clc1ccc([C@@H](N[C@@H](c2ccccc2)c2ccccc2)c2ccccc2)cc1', 'Clc1ccc(Br)cc1NCc1ccncc1', 'Clc1ccc(Br)cc1NCCCc1nnc2ccccn12', 'Clc1ccc(Br)cc1NCC(=O)N1CCCC1', 'Clc1ccc(Br)cc1NCCc1c[nH]c2ccccc12', 'Clc1ccc(Br)cc1NCC1(C2CC2)CC1', 'Clc1ccc(Br)cc1NCC[S@@](=O)Cc1ccccc1', 'Clc1ccc(Br)cc1NCCOCCOCn1ccccc1=O', 'Clc1ccc(Br)cc1NCCN1CCOCC1', 'Clc1ccc(Br)cc1NCC[C@@H]1CCCC[NH2+]1', 'Clc1ccc(Br)cc1NCCn1cccn1', 'Clc1ccc(Br)cc1NCCS(=O)(=O)c1ccccc1', 'Clc1ccc(Br)cc1NCC[C@H](O)c1ccccc1F', 'Clc1ccc(Br)cc1NCC[NH+]1CCCC1', 'Clc1ccc(-c2nonc2)nc1C(F)(F)F', 'Clc1ccc(Cc2ncc(C(F)(F)F)s2)cc1', 'Clc1ccc(Cc2nccc(CC[NH+]3CCOCC3)n2)cc1', 'Clc1ccc(Cc2ncco2)cc1O', 'Clc1ccc(Cc2nccnc2C(=O)NNC(=O)C2CC2)cc1', 'Clc1ccc(Cc2nccs2)nc1', 'Clc1ccc(Cc2ncc[nH]2)cc1F', 'Clc1ccc(CC[C@@H](Cl)c2cccc(Cl)c2Cl)cc1', 'Clc1ccc(CC[C@@H]2CCC[NH+](CC3CC3)C2)c(O)c1', 'Clc1ccc(Br)c(Sc2ccc(OCc3ccccc3)cn2)c1', 'Clc1ccc(Br)c(Sc2nccnc2N2CCNC2=O)c1', 'Clc1ccc(Br)c(-n2nc(Br)ccc2=O)c1O', 'Clc1ccc(-c2nnnn2/N=C/c2ccc(Cl)cc2Cl)cc1', 'Clc1ccc(-c2nonc2[S-])nc1', 'Clc1ccc(-c2nonc2[S-])nn1', 'Clc1ccc(Br)c([C@H]2CC[NH2+]C[C@@H]2c2cccc(Br)c2)c1', 'Clc1ccc(Br)c([C@H]2c3ccccc3C=CN2C(=O)c2ccc(O)cc2)c1', 'Clc1ccc(Br)c([C@H]2OCC[C@@H]2CO)c1O', 'Clc1ccc(Br)c([C@H]2[C@H]3C=C4C[C@@H](CSC4)[C@H]3OC(=O)C[C@@H]2C(=O)[O-])c1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1-c1nonc1NCc1ncc[nH]1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1Nc1cccc(Br)c1', 'Clc1ccsc1Nc1ncnc(CO)c1-n1cccc1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1-c1nnon1', 'Clc1ccsc1-c1nnco1', 'Clc1ccsc1-c1nnnn1-c1ccc(F)c(Cl)c1', 'Clc1ccsc1N[C@@H]1CCOC2(CCOCC2)C1', 'Clc1ccsc1N[C@@H]1c2ccccc2C[C@@H]1O', 'Clc1ccc([C@@H](O)[C@@H](Cc2cccc(F)c2)c2cccnc2)cc1', 'Clc1ccc([C@@H](O)[C@@H]2[C@@H]3CC[C@H](C3)[C@@H]2O)cc1', 'Clc1cc(Br)c([O-])c(C(=O)N2CCC(c3nccn3CC3CC3)CC2)c1', 'Clc1cc(Br)c([N+](=O)[O-])cc1Br', 'Clc1cc(Br)c(NCc2ccncc2)nc1Cl', 'Clc1cc(Br)c(N/C=C/c2ccccc2)c2[nH]ccc12', 'Clc1cc(Br)c(Nc2ncnc3ccccc23)cc1/C=N/c1ccccc1', 'Clc1cc(Br)c(N2CCN(Cc3cscn3)CC2)cc1F', 'Clc1cc(Br)c(CCNC(=O)[C@@H](Cc2ccccc2)[NH+]2CCN(c3ccccc3F)CC2)nc1C1CC1', 'Clc1cc(Br)c(C(F)(F)F)cc1OC(=O)C1CC1', 'Clc1cc(Br)c(Cc2ccccc2)c2c1COC2', 'Clc1cc(Br)c(C2=NC[C@@H]2CN2CCO[C@H](CCO)C2)cc1Cl', 'Clc1cc(Br)c(COC2CC2)cn1', 'Clc1cc(Br)c(CN2CCO[C@@H](c3ccccc3)C2)cc1Br', 'Clc1cc(Br)c(C[NH+]2CCN([C@@H]3CCC[C@H]3O)CC2)cn1', 'Clc1cc(Br)c(C[NH3+])nc1NC[C@@H]1CCCO1', 'Clc1cc(Br)c(C[NH2+][C@H]2CCc3nnsc32)cc1F', 'Clc1cc(NSc2nc(-c3cccs3)nc3c2CCCC3)ccn1', 'Clc1cc(NSc2ncc(-c3ccccc3Cl)n2C2CC2)cc2c1CCCC2', 'Clc1cc(NSc2nc3ccccc3s2)ccc1Br', 'Clc1cc(NSc2ncnc3ccccc23)c2ccccc2n1', 'Clc1cc(Nc2c([N+](=O)[O-])cc(Cl)cn2)ccc1F', 'Clc1cc(Nc2c(cnn2Cc2ccco2)c2ccccc2)ccc1[N+](=O)[O-]', 'Clc1cc(Nc2c(Cl)cccc2Cl)ncn1', 'Clc1cc(Nc2c([O-])n3ccccc3n2)ccc1Br', 'Clc1cc(NSc2cccs2)ccc1OS(=O)(=O)c1ccccc1', 'Clc1cc(NSc2ccc(Cc3nc[nH]n3)o2)ccc1OCc1ccc(Cl)cc1', 'Clc1cc(NSc2ccc3nccnc3n2)ncn1', 'Clc1cc(NSc2cccc(Cl)c2Cl)ccc1OC(F)(F)F', 'Clc1ccc(COCc2nnc(-c3ccccc3OC(F)(F)F)o2)cc1', 'Clc1ccc(COc2ncc(C[NH+]3CCCC3)[nH]2)cc1', 'Clc1ccc(COc2ncc3cccc(Br)c3c2)cc1', 'Clc1ccc(COc2ncccc2F)cc1Br', 'Clc1ccc(COc2nccnc2Cc2ccccc2)cc1', 'Clc1ccc(C(=O)c2ccccc2)cc1', 'Clc1ccc(C(=O)OC[C@@H](O)c2cccc(Cl)c2)cc1', 'Clc1ccc(C(=O)Oc2ccc(F)c(F)c2)cc1', 'Clc1ccc(C(=O)O[C@@H]2C[C@@H](c3ccccc3)[NH+](Cc3ccccc3F)C2)cc1', 'Clc1ccc(Cc2cc[nH]n2)c(C[NH2+]C2CC2)c1', 'Clc1ccc(C2CCC2)c(NCc2ccco2)c1O', 'Clc1ccc(C2CCN(C(=O)C(F)(F)F)CC2)cc1', 'Clc1ccc(C2CC[NH+](C[C@@H](O)c3cnccn3)CC2)cc1', 'Clc1ccc(C2CC2)cc1F', 'Clc1ccc(-c2ccccc2Cl)cc1', 'Clc1ccc(-c2ccccc2Cl)nc1C1CC1', 'Clc1ccc(-c2cccs2)c2c1C[C@@H](Cl)[C@H]2Br', 'Clc1ccc(-c2cccs2)c2nccnc12', 'Clc1ccc(C(F)(F)F)c(NCc2ccc(F)cc2Br)c1', 'Clc1ccc(C(F)(F)F)cc1N1CCCC[C@H]1CN1CCOCC1', 'Clc1ccc(C(F)(F)F)cn1', 'Clc1ccc(COC[C@@H](O)C[NH+]2CCCCC2)cc1', 'Clc1ccc(COC[C@@H]2CCCN2C(=O)c2ccccc2Cl)cc1', 'Clc1ccc(-c2cc(Cl)ccc2F)cn1', 'Clc1ccc(-c2cc(Cl)ncn2)cc1', 'Clc1ccc(-c2cc([N+](=O)[O-])nc(N3CCOCC3)n2)cn1', 'Clc1ccc(-c2cc(F)c(Cl)cc2N)cc1', 'Clc1ccc(-c2cc(F)c3[nH]ccc3c2)cc1F', 'Clc1ccc(-c2cc(F)ccc2O)cc1N', 'Clc1ccc(-c2ccccc2N2CCC[NH2+]C[C@H]2CS(=O)(=O)[O-])cc1', 'Clc1ccc(-c2ccccc2N2[C@H](Cl)[C@H](c3ccccc3)C3=C2CCCC3)cc1', 'Clc1ccc(-c2ccccc2N2c3ccc(F)cc3C(=O)N2C2CCCCC2)cc1', 'Clc1ccc(-c2csc3[nH]ncc23)c(F)c1', 'Clc1ccc(-c2nnnn2Cl)cc1', 'Clc1ccc(-c2nnnn2-c2ccc(F)cc2)cc1', 'Clc1ccc(-c2nnnn2-c2ccccc2Br)cc1F', 'Clc1ccc(-c2nnnn2Cl)cn1', 'Clc1ccc(-c2nnnn2Cc2ccccc2)cc1', 'Clc1ccc(-c2nnnn2/N=C/c2cccc([N+](=O)[O-])c2)cc1', 'Clc1ccc(-c2nnnn2/N=C\\c2ccccc2Cl)cc1', 'Clc1ccc(-c2nnnn2/N=C2\\S[C@H]3OCCC[C@@H]13)CC(=O)N2Cc1ccccc1', 'Clc1ccc(C[NH+](C[C@@H]2CCOC2)C[C@H]2CCCO2)cc1', 'Clc1ccc(-c2csc3[nH]ncc3c2=O)cn1', 'Clc1ccc(-c2csc3[nH]ncc23)cc1F', 'Clc1cc(Cl)nnc1-c1ccccc1', 'Clc1cc(Cl)nnc1-n1cnc2ccccc21', 'Clc1cc(Cl)nnc1OCc1ccco1', 'Clc1cc(Cl)nnc1Oc1ccc(F)cc1NCc1ccccc1', 'Clc1cc(Cl)nnc1O[C@H]1CC[C@@H]([NH2+]CC(F)(F)F)C1', 'Clc1cc(Cl)nnc1O/C=C(/CC#N)c1ccc(Cl)cc1', 'Clc1ccc(-c2cc(-n3cccn3)nc(-c3cc(F)ccc3F)n2)cc1', 'Clc1ccc(-c2cc([N+](=O)[O-])ccc2I)cc1F', 'Clc1ccc(-c2cc(Cl)cs2)cc1F', 'Clc1ccc(-c2cc(Cl)c(Cl)cn2)cc1', 'Clc1ccc(-c2cc(Cl)ccc2Cl)cc1', 'Clc1ccc(-c2cc(Cl)cn2CCO)cc1', 'Clc1ccc(-c2cc([C@H](Nc3cncc(F)c3)Nc3ccc(F)cc3F)[nH]c2)cc1', 'Clc1ccc(-c2cc([C@H](Cl)Cc3ccc(F)cc3)[nH]n2)cc1', 'Clc1ccc(-c2cc([C@H]([NH3+])C3CCCC3)cnc2C#N)cc1', 'Clc1ccc(-c2csc3[nH]cnc32)cc1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1Cl', 'Clc1ccc(-c2csc3[nH]nnc23)cn1', 'Clc1ccc(-c2cccnc2N=[N+]2CCCC2)cc1', 'Clc1ccc(-c2cccnc2N=C2CCOCC2)cc1', 'Clc1ccc(-c2csc3[nH]cc(-c4noc5ccccc45)c23)cc1', 'Clc1ccc(-c2csc3[nH]ccc23)cc1', 'Clc1ccc(-c2csc3[nH]cnc32)c(C(=O)Nc2ccc(Br)cc2)c1', 'Clc1ccc(-c2csc3[nH]cnc23)cc1', 'Clc1ccc(-c2csc3[nH]cc(Br)c23)c(Br)c1', 'Clc1ccc(-c2csc3[nH]cc(C(F)(F)F)c-23)cc1', 'Clc1ccc(-c2csc3[nH]cnc32)cc1Cl', 'Clc1ccc(-c2csc3[nH]ncc23)cc1', 'Clc1ccc(-c2csc3[nH]ncc3c2=O)cc1', 'Clc1ccc(-c2csc3[nH]ncc32)cc1F', 'Clc1ccc(-c2n[nH]cc2CC2CC2)cc1', 'Clc1ccc(-c2n[nH]cc2C(F)(F)F)cc1', 'Clc1ccc(-c2n[nH]cc2C2CC2)cc1', 'Clc1ccc(-c2n[nH]cc2COc2ccc(F)cc2)cc1', 'Clc1ccc(-c2n[nH]cc2CN2CC[C@@H](Nc3ccccc3F)C2)cc1', 'Clc1ccc(-c2n[nH]cc2Cn2cnc3ccc(F)cc32)cc1', 'Clc1ccc(-c2n[nH]cc2CSc2n[nH]c(=O)n2CCc2ccccc2)cc1', 'Clc1ccc(-c2n[nH]cc2C[NH+]2CCc3[nH]c4ccccc4c3C2)cc1', 'Clc1ccc(-c2csc3[nH]ccc23)nc1O', 'Clc1ccc(-c2cccs2)nn1', 'Clc1ccc(-c2cccs2)[nH]c1=O', 'Clc1ccc(-c2cc(-c3ccccc3O)nnc2-c2ccccc2)cc1', 'Clc1ccc(-c2cc(O[C@H](C(=O)[O-])C3CC3)[nH]n2)cc1', 'Clc1ccc(-c2cc(O[C@H]3CCCOC3)nc(-c3ccc(Cl)cc3)n2)cc1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'Clc1ccc(-c2nonc2N2CCC[C@H](C(=O)Nc3ccc4c(c3)CCC4)C2)cc1', 'Clc1ccc(-c2nonc2NCCN2CCOCC2)cn1', 'Clc1ccc(-c2nonc2NC(=O)Cc2c(Cl)cccc2Cl)cc1', 'Clc1ccc(-c2nonc2NCc2nc(-c3ccncc3)no2)cc1', 'Clc1ccc(-c2nonc2NC2CCCC2)o1', 'Clc1ccc(-c2nonc2NC[C@@H]2CCCSC2)c(F)c1', 'Clc1ccc(-c2nonc2NC[C@H]2CCCO2)cc1', 'Clc1ccc(-c2nnnn2C2CC2)cc1', 'Clc1ccc(-c2nnnn2C2(C(=O)[O-])CCCCC2)cc1', 'Clc1ccc(-c2nnnn2-c2cc([N+](=O)[O-])ccc2OC(F)F)cc1', 'Clc1ccc(-c2nnnn2-c2cc(C(F)(F)F)cs2)cc1', 'Clc1ccc(-c2nnnn2-c2cc(OCc3cccnc3)ccc2[N+](=O)[O-])cc1', 'Clc1ccc(-c2nnnn2-c2cc(Nc3ccccc3)ccn2)cc1', 'Clc1ccc(-c2nnnn2-c2cc(F)ccc2F)cc1', 'Clc1ccc(-c2nnnn2-c2cc(Cl)ccc2Cl)cc1', 'Clc1ccc(-c2nnnn2Cl)cc1', 'Clc1ccc(-c2nnnn2Cl)cc1O', 'Clc1ccc(-c2nnnn2Cl)cc1Cl', 'Clc1ccc(-c2nnnn2Cl)cc1F', 'Clc1ccc(-c2n[nH]c3ccccc23)[nH]1', 'Clc1ccc(-c2n[nH]c3c2[C@H](c2ccccc2)NC3=O)c(O)c1Cl', 'Clc1ccc(-c2nonc2)cc1F', 'Clc1ccc(-c2nonc2)nn1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'Clc1ccc(-c2csc3s/cnc23)nc1', 'Clc1ccc(-c2nn(-c3ccccc3)cc2F)cc1', 'Clc1ccc(-c2nn(-c3cc(Cl)ccc3Br)nc2N2CCOCC2)cc1', 'Clc1ccc(-c2nn(-c3ccccc3)cc2C(F)(F)F)cc1', 'Clc1ccc(-c2nn(-c3ccc[nH]c3=O)cc2F)cc1Cl', 'Clc1ccc(-c2nn(-c3ccc(F)cc3)cn2)cc1', 'Clc1ccc(-c2nn(-c3ccccc3)[nH]c2=O)cc1F', 'Clc1ccc(-c2nn(-c3cc([N+](=O)[O-])ccc3Cl)nn2)cc1', 'Clc1ccc(-c2nn(-c3cc(C(F)(F)F)ccc3Br)[nH]c2=O)cc1', 'Clc1ccc(-c2nn(-c3cc(F)ccc3F)c(=S)s2)cc1', 'Clc1ccc(-c2nn(-c3cc(Br)cc(F)c3)cn2)cc1', 'Clc1ccc(-c2n[nH]c3[nH]c[n+]([O-])c23)cc1', 'Clc1ccc(-c2n[nH]c3[nH]cnc23)cc1F', 'Clc1ccc(-c2nonc2Cn2cccc(Br)c2=O)cc1', 'Clc1ccc(-c2nonc2)cn1', 'Clc1ccc(-c2nn([C@@H](O)[C@@H](CC(=O)NC3CC3)C(C#N)=C3CCCCC3)n2Cc2ccccc2)cc1', 'Clc1ccc(-c2nn([C@@H](c3ccccc3)[NH+]3CCC(OC(F)F)C3)cc2F)cn1', 'Clc1ccc(C[NH2+][C@@H](CO)c2ccco2)cc1', 'Clc1ccc(C[NH2+][C@@H]2CCCC[C@H]2CN2CCOCC2)cc1', 'Clc1ccc([C@@H]([C@@H]2Cc3cnccc3S2)N2CCOCC2)cc1', 'Clc1ccc(Br)cc1Sc1nc(NC2CCCCCC2)n[nH]1', 'Clc1ccc(Br)cc1Sc1ncn[nH]1', 'Clc1ccc(Br)cc1Sc1ncccc1OCc1ccoc1', 'Clc1ccc(Br)cc1Sc1nc2ccccc2n1Cc1ccccc1Cl', 'Clc1ccc(-c2cccnc2Cl)c(-n2ccnc2)n1', 'Clc1ccc(-c2cccnc2Cl)cc1', 'Clc1ccc(-c2cccnc2Cl)cn1', 'Clc1ccc(-c2ccccc2)c(N/C=C2\\C(=O)N(c3ccc(Cl)cc3)C2=O)c1', 'Clc1ccc(-c2ccccc2)cc1O', 'Clc1ccc(-c2cccs2)nc1', 'Clc1ccc(-c2cccs2)nc1CNc1ccccc1', 'Clc1ccc(-c2cccs2)nc1[N-]S(=O)(=O)c1ccc(Cl)cc1', 'Clc1ccc(-c2cccs2)nc1OC1CCN(C(=O)c2ccccc2)CC1', 'Clc1ccc(-c2cccs2)nc1NCC[NH+]1CCCCC1', 'Clc1ccc(-c2cccs2)nc1Cl', 'Clc1ccc(-c2csc3[nH]nnc23)s1', 'Clc1ccc(-c2cc(SCc3nc4ccc(F)cc4o3)n[nH]2)cc1', 'Clc1ccc(-c2cc(SC[C@@H](CCl)N3CCOCC3)ncn2)cc1', 'Clc1ccc(-c2cc(SC[C@H]3CCCO3)ncn2)cc1', 'Clc1ccc(-c2cc([N+](=O)[O-])ccc2OCC(F)F)cc1F', 'Clc1ccc(-c2cc(-c3cnc([C@H]4CCCO4)nn3)[nH]n2)cc1Cl', 'Clc1ccc(-c2csc3s/cnc23)c(Br)c1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'Clc1ccc(-c2ccccc2Cl)c(-n2cnnn2)c1', 'Clc1ccc(-c2ccccc2Cl)cc1', 'Clc1ccc(-c2cccnc2Sc2ccc(C[NH+]3CCCC3)cc2)cc1', 'Clc1ccc(-c2csc3s/cnc23)n2c1nc1ccccc12', 'Clc1ccc(-c2csc3s/cnc23)nc1', 'Clc1ccc(-c2csc3s/cnc23)nn1', 'Clc1ccc(Br)cc1-c1cn(-c2c(F)cccc2F)nc1N1CCCCC1', 'Clc1ccc(Br)cc1-c1cn(-c2ccccn2)nc1C1CC1', 'Clc1ccc(Br)cc1-c1ccs/c(=O)c1C(F)(F)F', 'Clc1ccc(Br)cc1-c1ccsc1', 'Clc1ccc(Br)cc1-c1ccsn1', 'Clc1ccc(C(CO)Oc2cccc(Br)c2)cc1', 'Clc1ccc(C(=O)NCCCN2CCO[C@@H](c3ccccc3)C2)cc1', 'Clc1ccc(C(=O)Nc2ccc(Oc3c(F)cccc3F)o2)cc1', 'Clc1ccc(C(=O)N2CCCC2)c2c1cnn2-c1ccc(Cl)cc1Cl', 'Clc1ccc(C(=O)NNC(=O)N2CCCCCC2)cc1F', 'Clc1ccc(C(=O)N[C@@H]2CCC(Cn3cc[nH+]c3)CCOCC2)cc1', 'Clc1ccccc1[C@H](Cl)c1ccc(Cl)cc1F', 'Clc1ccccc1[C@H](NC1CC1)c1sccc1Br', 'Clc1ccccc1[C@H](N)C(=O)[O-]', 'Clc1ccccc1[C@H](Nc1cccc(F)c1)C(F)(F)F', 'Clc1ccccc1[C@H](NNC(=O)C1CCC1)c1ccccc1', 'Clc1ccccc1[C@H](NS(=O)(=O)c1ccc(Cl)cc1)c1cccc2ccccc12', 'Clc1ccccc1[C@H](N[C@H](c1ccccc1)C1CC1)C1CCCC1', 'Clc1ccccc1[C@H](Cc1ccco1)N1CC[NH+](C2CCCC2)CC1', 'Clc1ccccc1[C@H](CO)NCc1ccc([N+](=O)[O-])c([N+](=O)[O-])c1', 'Clc1ccccc1[C@H](CNC(=O)Nc1ccc(Br)cc1)N1CCOCC1', 'Clc1ccccc1[C@H](C#N)C(=O)Cc1cccnc1', 'Clc1ccccc1[C@H](CCl)c1c(F)cccc1F', 'Clc1ccccc1[C@H](C[NH+](Cc1ccccc1)C1CCCC1)Nc1cc(F)ccc1Br', 'Clc1ccccc1[C@H]1[C@@H](Cc2ccccc2)c(=O)n(-c2ccccc2)N1c1ccc2c(c1)OCO2', 'Clc1ccccc1[C@H]1[C@@H]2O[C@@H](COC(=O)c3ccccc3)[C@H]2C(=O)N1c1ccc(Cl)s1', 'Clc1ccccc1[C@H]1[C@H]2NC(=S)N=C2S1', 'Clc1ccccc1NC(=O)[C@H](Cc1c(Cl)cccc1Cl)Nc1ccccc1O', 'Clc1ccccc1Nc1cn(CCc2ccccc2)nc1CSc1ccccc1', 'Clc1ccccc1Nc1cncc(C(F)(F)F)n1', 'Clc1ccccc1NCc1ccccc1N1C(=O)CCc2ccccc21', 'Clc1ccccc1NC1=NN([C@@H]2CCS(=O)(=O)C2)C1', 'Clc1ccccc1NC1CCN(c2ccccc2OCc2ccc(F)cc2)CC1', 'Clc1ccccc1NC1([N+](=O)[O-])CCCCC1', 'Clc1ccccc1NC12CC3CC(CC(C3)C1)C2', 'Clc1ccccc1[C@H]1NC(=O)COc2c1ccc1c2cccc1OCc1ccco1', 'Clc1ccccc1[C@H]1Nc2c(cccc2Cl)C(=O)N1C1CCCCC1', 'Clc1ccccc1[C@H]1N=NC[C@H]2COCCS[C@@H]21', 'Clc1ccccc1[C@H]1NN[C@H](c2ccco2)N1O', 'Clc1ccccc1[C@H]1N/C(=C/c2ccc(-c3ccco3)cc2)N1c1ccc(F)cc1', 'Clc1ccc(-c2nonc2CNc2cc(Cl)ccc2Cl)cn1', 'Clc1ccc(-c2nonc2CN2CCN(c3ccccc3Cl)CC2)cc1', 'Clc1ccc(-c2nonc2)nc1', 'Clc1ccc(-c2nonc2)nc1C1CC1', 'Clc1ccc(-c2nonc2)nc1O[C@@H]1CCOC2(CCCCC2)C1', 'Clc1ccc(-c2nonc2)nc1Nc1cc(Br)ccc1OC(F)F', 'Clc1ccc(-c2nonc2)nc1-c1ccc(Br)cc1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc([C@@H]([NH2+][C@H]2CC[C@@H]([NH3+])C2)C2CC2)cc1', 'Clc1cc(/C=C/NC2CC[NH+](Cc3ccccc3)CC2)ccc1F', 'Clc1cc(/C=C/Nc2ccc(OCc3ccccc3)cc2)ccc1Cl', 'Clc1cc(/C=C/N2CCN(c3cccc([N+](=O)[O-])c3)CC2)c[nH]1', 'Clc1cc(/C=C/N=C(\\c2ccccn2)c2ccccc2)cc(Cl)c1', 'Clc1cc(CN(Cc2ccccn2)C2CC2)ccn1', 'Clc1cc(CN(c2c(Cl)cccc2Cl)SC2CCCC2)c2ncccn12', 'Clc1cc(CCl)nc2ccccc12', 'Clc1cc(CCl)n2cncc2n1', 'Clc1cc(CCl)n[n-]1', 'Clc1cc(CCl)nnc1-c1cnccn1', 'Clc1cc(CCl)n[nH]1', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1OS(=O)(=O)Cc1ccccc1F', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccccc1', 'Clc1cccc([S@@]C(=S)Nc2ccc3ccc(=O)n(Cc4ccccc4)c3c2)c1', 'Clc1cccc([S@@]C(CC2CC2)C2CC2)c1', 'Clc1cccc([S@@]C(Br)(CO)C(F)(F)F)c1', 'Clc1cccc(-c2cccc(I)c2)c1', 'Clc1cccc(-c2cnc(-c3ccc4ccccc4c3)o2)c1', 'Clc1cccc(-c2cn(CC(CC[NH3+])C3CCN(c4cc[nH+]cc4)C3)cn2)c1', 'Clc1cccc(-c2cncc(Br)n2)c1', 'Clc1cccc(-c2cnsc2)c1', 'Clc1cccc(Nc2cn(Cc3ccccc3)nc2C[NH+]2CCCCCC2)c1', 'Clc1cccc(Nc2cnccc2S(=O)(=O)/N=C/c2ccc(F)cc2)c1', 'Clc1cccc(Nc2cn[n+]([O-])cc2O[C@@H]2CCCCS2)c1', 'Clc1cc(/C=N/OCN(Cc2ccco2)CC2CC2)ccn1', 'Clc1cc(/C=N/O)c(I)c(C(F)(F)F)c1[N+](=O)[O-]', 'Clc1cc(/C=N/Oc2ccccc2Cl)ccc1OS(=O)(=O)c1ccccc1', 'Clc1cc(/C=N/O[C@@H]2CCCCNC2)cs1', 'Clc1cc(/C=N/O[C@H]2CC=CCC2)ccc1Cl', 'Clc1ccc(-c2cc(/C=C3\\C(=O)N(Cc4ccco4)C3=O)oc2[O-])cc1', 'Clc1ccc(-c2cc(/C([O-])=NO[C@@H]3CCC[C@@H](CO)[C@H]3NC(F)(F)F)nn2C2CCCCC2)cc1', 'Clc1ccc(-c2csc3[nH]cnc23)cc1Cl', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1-n1nnc(C(F)(F)F)n1', 'Clc1ccc(Br)cc1-n1nncc1C[NH+]1CCCCC1', 'Clc1ccc(Br)cc1-n1nnc2ccc([N+](=O)[O-])cc21', 'Clc1ccc(-c2csc3[nH]c(NC[C@@H]4CCCO4)cc23)cc1', 'Clc1ccc(-c2csc3[nH]c(Nc4ccccc4)cc23)cc1', 'Clc1ccc(-c2csc3[nH]c(N[C@@H]4CCSCC4)cc23)cc1', 'Clc1ccc(-c2csc3[nH]c(N[C@H]4CCO[C@@H](CO)C4)nc23)cc1', 'Clc1ccc(-c2csc3[nH]c(N4CCCC4)nc23)cc1Br', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O[C@H]1CCCN(C(c2ccccc2)C(F)(F)F)C1', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccccc1O', 'Clc1ccc(-c2n[nH]c3cc(C(=O)NCC[NH+]4CCCCCC4)ccc23)cc1', 'Clc1ccc(-c2n[nH]c3ccc(OC(F)(Cl)F)cc23)cc1', 'Clc1cccc2cccnc12', 'Clc1cccc2ccc(COCCNC(=O)CC[C@@H]3CCC[NH+](CC4CC4)C3)nc12', 'Clc1cccc2ccc(OCCSc3nnnn3-c3ccccc3)c2c1', 'Clc1cccc2ccc(NCc3cc(F)c(F)cc3F)cc12', 'Clc1cccc2ccc(S(=O)(=O)NCC(c3ccccc3)c3ccc(Cl)cc3)nc21', 'Clc1cccc2ccc(Cl)cc12', 'Clc1cccc2ccc(Br)cc12', 'Clc1cccc2ccc(/C=C/O)cc12', 'Clc1ccc(Cc2noc(CCC3CCCCC3)n2)o1', 'Clc1ccc(Cc2noc(C(=O)N3CCCCC3)n2)cc1', 'Clc1ccc(Cc2noc(Cc3ccncc3)n2)cc1', 'Clc1ccc(Cc2noc(COc3ccccc3)n2)cc1', 'Clc1ccc(Cc2noc(C3CC3)c2[N+](=O)[O-])cc1', 'Clc1ccc(Cc2noc(C[C@@H]3CCCN3C(=O)c3ccccc3)n2)cc1', 'Clc1ccc(Cc2noc(C[NH+]3CCN(c4ccc(Cl)cc4)CC3)n2)cc1', 'Clc1ccc(Cc2noc(C[NH2+]C3CC3)c2)cc1', 'Clc1ccc(CNc2cccc(Cl)c2)cc1', 'Clc1ccc(CNc2sc3c(n2)CN(Cc2ccc4c(c2)OCO4)CC3)cc1', 'Clc1ccc(CNc2ccc3c(c2)OCO3)cc1', 'Clc1ccc(CN[C@@H](C(=O)c2ccccc2)c2ccccc2)cc1', 'Clc1ccc(CN[C@@H](c2nc3ccccc3s2)c2ccccc2)cc1', 'Clc1ccc(CNc2c[nH]c(C(=O)[O-])c2)o1', 'Clc1ccc(CCOC2CCOCC2)cc1', 'Clc1ccc(CCO/N=C(\\C)CCc2ccccc2)cc1', 'Clc1ccc(CCO)c(NCc2ccccc2Br)c1', 'Clc1ccc(CCOc2ccc3ccccc3c2)cc1', 'Clc1ccc(CCNCc2cc(F)cc3c(O)cccc23)cc1', 'Clc1ccc(CCN(Cc2ccco2)C[C@@H]2C=CCCC2)cc1', 'Clc1ccc(CCN[C@H]2CC(=S)N(C3CCCCCC3)C2)cc1', 'Clc1ccc(CCN2CCN(c3cccc[nH+]3)CC2)cc1', 'Clc1ccc(CO)c(C(=O)N2CCCCC2)c1', 'Clc1ccc(CO)c(Oc2ccccc2Br)n1', 'Clc1ccc(CO)c(N2CCN(Cc3ccco3)CC2)c1', 'Clc1ccc(CO)c([C@@H]2CSCC[NH2+]2)c1', 'Clc1ccc(CO)c(-c2cc(Cl)cn3ncnc23)c1', 'Clc1ccc(CO)c(SCc2nc3ncccc3o2)c1', 'Clc1ccc(CO)c(Cl)c1', 'Clc1ccc(CO)c([C@H]2CSCCO2)c1', 'Clc1ccc(CO)c(/C(Cl)=[N+]([O-])/C=C/c2ccccc2)c1', 'Clc1ccc(C2SCCCCCC2)cc1F', 'Clc1ccc(C2SCCC[S@@]2=O)cc1', 'Clc1ccc(C2SCCC2)cc1Cl', 'Clc1ccc(C2SCCCN2CCSC2CC2)cc1', 'Clc1ccc(C2SCCCS2)cc1', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1OS(=O)(=O)CCO', 'Clc1ccc(Br)cc1OS(=O)(=O)C(F)(F)F', 'Clc1ccc(Br)cc1OS(=O)(=O)Cc1ccccc1', 'Clc1ccc(Br)cc1OS(=O)(=O)C1CC1', 'Clc1ccc(Br)cc1OS(=O)(=O)C[C@@H]1COc2ccccc2O1', 'Clc1ccccc1-c1cscc1Br', 'Clc1ccccc1-c1cscc1-c1cc(=O)[nH]c2ccccc12', 'Clc1ccccc1-c1cscc1C(F)(F)F', 'Clc1ccccc1-c1cscc1F', 'Clc1ccccc1-c1cscc1-c1ccccc1', 'Clc1ccccc1-c1cscc1-n1ccc2ccccc21', 'Clc1ccccc1-c1cscc1F', 'Clc1ccccc1-c1cncc(CN2CCC[C@H]2c2ccccc2F)n1', 'Clc1ccccc1-c1cncc2c1CN2C(F)(F)F', 'Clc1ccccc1-c1cnccc1N[C@@H]1CCN(c2cccc(Cl)c2)C1=O', 'Clc1ccccc1-c1cnccn1', 'Clc1ccccc1-c1cscc1C(=O)Nc1ccc(C(=O)[O-])cc1', 'Clc1ccccc1-c1cscc1C[NH3+]', 'Clc1ccccc1-c1cscc1C[NH2+]Cc1ccccc1', 'Clc1ccccc1-c1cscc1Cn1cncn1', 'Clc1ccccc1Oc1cc(SC(F)F)nc2cccnc12', 'Clc1ccccc1Oc1cc(S(=O)(=O)N2CCN(CC(F)(F)F)CC2)ccc1Cl', 'Clc1ccccc1Oc1cc(Sc2nc3c(s2)CCCC3)cs1', 'Clc1ccccc1-c1nnc(CSc2ncccn2)o1', 'Clc1ccccc1-c1nncn1[C@@H]1CC[C@@]2(CCCN(Cc3ccc(F)cc3)C2)C1', 'Clc1ccccc1-c1nncc(Cl)c1F', 'Clc1ccccc1-c1nnc2n1NCC[C@@H]1C[C@@H]1C2', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12', 'Clc1cccc2[nH]ncc12\n', 'Clc1cccc2[nH]ncc12\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n', 'Clc1cccc2[nH]nc(SCc3ccsc3)c12', 'Clc1cccc2[nH]nc(S(=O)(=O)[N-]c3ccc(-n4cccn4)c([N+](=O)[O-])c3)c12', 'Clc1cccc2[nH]nc(Sc3cccnc3)c12', 'Clc1cccc2[nH]nc(S[C@H]3CC[NH2+]C[C@@H]3Cc3ccccc3)c12', 'Clc1cccc2[nH]nc(S/C=C/C3CCCCC3)c12', 'Clc1ccccc1OCc1csc(-c2ccccc2)n1', 'Clc1ccccc1OCc1cc(=O)n2c(n1)SCCN2', 'Clc1ccccc1Cc1c(Cl)ncnc1N[C@@H](C(=O)c1ccccc1)[C@H]1CCCO1', 'Clc1ccccc1Cc1cccc(OC2CC[NH2+]CC2)c1', 'Clc1ccccc1Cc1cn(CC(=O)[O-])c2ccccc12', 'Clc1ccccc1Oc1cnc2scc(-c3ccccc3)c2c1', 'Clc1ccccc1Oc1cnc2ccccc2n1', 'Clc1ccccc1Oc1cnc2ncnc(-c3ccccc3)c2c1', 'Clc1ccccc1OCc1csc(C(=O)N[C@@H](CCS2CCOC2)C(F)(F)F)c1', 'Clc1ccccc1N1N=CC(c2ccccc2)c2ccccc21', 'Clc1ccccc1N1N=C(c2ccco2)CC1', 'Clc1ccccc1N1N=Cc2ccc(Cl)cc2C1', 'Clc1ccccc1N1N=C2SCC[C@@H]2C1=O', 'Clc1ccccc1N1N=CN(S(=O)(=O)c2ccc(Cl)c(Cl)c2)CC1', 'Clc1ccccc1N1N=C/c2ccccc2[C@H]1c1ccccc1F', 'Clc1ccccc1CSCCc1ccccc1', 'Clc1ccccc1CSCc1cccc(Br)c1', 'Clc1ccccc1CSC1=NC2=CC=N[C@@H]2S1', 'Clc1ccccc1CSC[C@@H]1CCCCC[C@@H]1C(=O)[O-]', 'Clc1ccccc1CSC[C@H](O)COc1ccc(F)cc1', 'Clc1ccccc1CSC/C=C/C(=O)[O-]', 'Clc1cccc([C@@H](SCc2ccccc2)C(F)(F)F)n1', 'Clc1cccc([C@@H](Sc2ccccc2)[C@H]2CCCO2)c1', 'Clc1ccc([C@@H](Br)[C@@](O)(c2ccccc2)c2ccccc2)cc1', 'Clc1ccc([C@@H](Br)[C@@]23CC[C@H](C[C@@H]2C)C3=O)cc1Cl', 'Clc1cc(Cl)nn1/N=C\\c1ccc2c(c1)OCO2', 'Clc1cc(Cl)nn1/N=C\\c1ccc(Cl)cc1', 'Clc1cc(Cl)nn1/N/c1cccc([N+](=O)[O-])c1', 'Clc1cc(F)cc(NCC2(O)CC[NH+](CC[NH+]3CCCCC3)CC2)c1', 'Clc1cc(F)cc(Nc2cc(F)ccc2F)c1', 'Clc1cc(F)cc(N2CC[C@@H]([NH2+][C@H]3CCNC3=O)C2)c1', 'Clc1cc(F)cc(NNC(=O)COc2cccc(Br)c2)c1', 'Clc1cc(F)cc(N[C@@H]2CCCC[C@@H]2O)c1', 'Clc1cc(F)cc(N[C@H]2CCN(c3ccccc3)C2)c1', 'Clc1cc(F)cc(N/N=C2\\N=NC3=C2CCc2ccccc23)c1', 'Clc1cccc2nc(N3CCC[C@@H](CC4CC[NH+](C5CCCCC5)C4)CC3)nn12', 'Clc1cccc2nc(Nc3ccc4c(c3)OCO4)nc2c1', 'Clc1cccc2nc(N[C@@H]3CCO[C@]4(CCOC4)C3)oc12', 'Clc1ccc(-c2ccccc2)nc1SCC1CCCC1', 'Clc1ccc(-c2ccccc2)nn1', 'Clc1ccc(-c2ccccc2CC2(O)COC2)cc1', 'Clc1ccc(-c2ccccc2CCO)cn1', 'Clc1ccc(-c2ccccc2CCn2cnc3ccccc32)cc1', 'Clc1ccc(-c2ccccc2CCSCc2ccc(F)cc2F)cc1', 'Clc1ccc(-c2cc[nH]n2)o1\n\n\n', 'Clc1ccc(-c2cccnc2N2CCC[C@@H]2C(F)(F)F)cc1', 'Clc1ccc(-c2cccnc2N2[C@@H](O)CCC[C@H]2OCc2ccccc2)cc1', 'Clc1ccc(-c2ccccc2[N-]C2=NC([N+](=O)[O-])C3=C2C(=O)CCC3)cc1', 'Clc1ccc(-c2ccccc2[N-]C[C@H](O)C[NH+]2CCCC2)cc1', 'Clc1ccc(CN[C@@H](CCO)C2CC2)cc1', 'Clc1ccc(CN[C@@H](C(=O)N2CCCC2)c2ccccc2)nc1', 'Clc1ccc(CN[C@@H](Cc2c[nH]c3ccccc23)C2CC2)cc1', 'Clc1ccc(CN[C@@H](CO)CC2CCCC2)cc1', 'Clc1ccc(CN[C@@H](C[C@@H]2CCCCO2)c2nccn2-c2ccccc2F)cc1', 'Clc1ccc(CN[C@@H](Cn2cnc3ccccc32)N2CCCC2)cc1', 'Clc1ccc(Cc2cccs2)nc1', 'Clc1ccc(Cc2ccco2)nc1', 'Clc1ccc(Cc2ccc(CCn3nc(-c4ccccc4O)o3)o2)cc1', 'Clc1ccc(Cc2cccc([N+](=O)[O-])c2)cc1', 'Clc1ccc(CN2N=C3c4ccccc4O[C@@H]3C2)s1', 'Clc1ccc(CN2NCCN(c3ccccc3C(=O)[O-])CC2)cc1', 'Clc1ccc(CC(=O)N[C@@H]2CCc3ccccc32)cn1', 'Clc1ccc(CCCC[NH2+][C@@H]2CCC[C@H](C(F)(F)F)C2)cc1', 'Clc1ccc(CCC(F)(F)F)cc1', 'Clc1ccc(CCC2C=[NH+][C@]3(CCO/C(=C/C(=O)c4ccc5c(c4)OCO5)C3)[C@H]2C)cc1', 'Clc1ccc(CCCN2CCc3[nH]c4ccccc4c3C2)cc1', 'Clc1ccc(CCC[NH+]2CCCC[C@H]2c2ccccc2)s1', 'Clc1ccc(CCC[NH2+]C[C@H]2CCC[C@@H]2c2ccccc2F)c(F)c1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1-c1cc(Br)ccn1', 'Clc1ccsc1-c1ccncc1', 'Clc1ccsc1-c1ccc(Cl)cc1', 'Clc1ccsc1-c1cc2cccc(F)c2nc1NC1COC1', 'Clc1ccsc1-c1csc(N2CCCCCC2)n1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1NSc1ncccn1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1SC[C@@H](O)c1cccc(Br)c1', 'Clc1ccsc1SC[C@@H]1CCC[NH2+]C1', 'Clc1ccc(Br)c(-n2nnc(C(F)(F)F)c2C2CC2)c1', 'Clc1ccc(Br)c(-n2nnnc2SCc2csc(-c3ccccc3)n2)c1', 'Clc1ccc(Br)c(S[C@H](C2CCCCC2)C2CC2)c1', 'Clc1ccc(Br)c(S[C@H]2CCOC3(CCCCC3)C2)c1', 'Clc1ccc(Br)c(NCC[NH+]2CCCC2)c1', 'Clc1ccc(Br)c(NC(=S)N(c2nccc([N+](=O)[O-])c2)[C@H]2CCSC2)c1', 'Clc1ccc(Br)c(NCc2cccnc2C2CC2)c1O', 'Clc1ccc(Br)c(NC2CCCCC2)c1', 'Clc1ccc(Br)c(NC[C@@H]2NCCC[NH+]2CC2CC2)c1', 'Clc1cc(Cl)c(CCCOc2ccccc2Cl)c(Br)c1Cl', 'Clc1cc(Cl)c(C(=O)Br)cc1Oc1ccc(F)cc1', 'Clc1cc(Cl)c(Cc2cccs2)c2ncc(-c3ccccc3)n12', 'Clc1cc(Cl)c(C2CC2)cc1N[C@@H]1CC[C@H](S(=O)(=O)/N=C/c2ccc3c(c2)OCO3)C1', 'Clc1cc(Cl)c(COC2CCCCC2)cc1O', 'Clc1cc(Cl)c(CN(Cc2ccccc2)C(F)(F)F)cc1O', 'Clc1cc(Cl)c(Cn2c(=O)[nH]c3ccccc32)c(Br)c1Br', 'Clc1cc(Cl)c(C[NH+]2CCC[C@H]2CN2CCOCC2)cc1Cl', 'Clc1cc(Cl)c(C[NH2+]CC2CCCCC2)cc1F', 'Clc1cc(Br)c2nc(Oc3ccccc3)cnc2n1', 'Clc1cc(Br)c2nc(OCC(=O)N3CCN(C(=O)Cn4ccc5ccccc54)CC3)ncc2c1', 'Clc1cc(Br)c2nc(OCc3ccccc3F)[nH]c2c1', 'Clc1cc(Br)c2nc(OC3CCCC3)[nH]c2c1', 'Clc1cc(Br)c2nc(OC[C@H]3CCCO3)ncc2c1O', 'Clc1cc(Br)cn1Nc1ncccc1F', 'Clc1cc(Br)cn1N1CC(c2ccc(CO)cc2)C1', 'Clc1cc(Br)cn1N=C\\c1ccc(Br)cc1', 'Clc1cc(Br)cn1N[C@@H]1C[C@H](O)CN1C(=O)Nc1cccc2ccccc12', 'Clc1cc(NC2=NC=CC2)ccc1O', 'Clc1cc(NC2CC[NH+](Cc3ccc4c(c3)OCCO4)CC2)ccc1F', 'Clc1cc(NC2(c3ccc(F)cc3)CCOCC2)cs1', 'Clc1cc(Cl)nn1/N=C\\c1cccc(Br)c1', 'Clc1cc(Cl)nn1/N=N\\N1CCN(c2ncc(-c3ccccc3)s2)CC1', 'Clc1cc(Br)c2nc(-c3ccc(Br)cc3)oc2c1', 'Clc1cc(Br)c2nc(-c3cccs3)[nH]c2n1', 'Clc1cc(Br)c2nc(-c3cc(Cl)cs3)oc2c1', 'Clc1cc(Br)c2nc(-c3ccco3)cc(-n3ccnc3)c2c1', 'Clc1cc(Br)c2nc(-c3ccncc3)ncc2c1', 'Clc1cc(Br)c2nc(-c3cc[nH]c3)[nH]c2c1', 'Clc1cc(N2CCc3ccccc3C2)ccc1O', 'Clc1cc(N2CC[C@H](Nc3ncccn3)C2)ncc1Cl', 'Clc1cc(N2CCOCC2)ccc1OCc1cc[nH+]c2ccccc12', 'Clc1cc(N2CCOc3ccccc32)nnc1-c1ccccc1', 'Clc1cc(N2CCO[C@@H](c3ccccc3F)C2)nc2ccccc12', 'Clc1cc(N2CCO[C@H](CO)C2)cc(Cl)c1Cl', 'Clc1cc(NSc2nnc(-c3ccc(Cl)cc3)n2Cc2ccc(F)cc2)ncn1', 'Clc1cc(NSc2nnnn2C2CCCC2)ccn1', 'Clc1cc(NSc2nnc(-c3ccccc3)n2Cc2ccco2)n[nH]1', 'Clc1cc(NSc2nncc3c(O)cccc23)no1', 'Clc1cc(NSc2nncn2C2CC2)ccc1Oc1ccccc1', 'Clc1cc(NSc2nncs2)ccn1', 'Clc1cc(/N=C/c2sc3ccccc3c2C[NH+]2CCCC2)cc(O)c1O', 'Clc1cc(Cl)nc2[nH]nc(-c3ccccc3)c12', 'Clc1cc(Cl)nc2ccc(C(=O)OC[C@@H]3CCCO3)cc12', 'Clc1cc(Cl)nc2ncnc(Nc3ccc4c(c3)OCO4)c12', 'Clc1cc(Cl)nnc1NCc1cnn([C@@H]2CCC[NH2+]C2)c1C1CC1', 'Clc1cc(Cl)nnc1Nc1ccc(Br)cc1', 'Clc1cc(Cl)nnc1N1CCCCC1', 'Clc1cc(Cl)nnc1N[C@@H]1CCOC2(CCCCC2)C1', 'Clc1cc(Cl)nnc1N[C@H]1C[C@H]2CC[C@@H](C1)[NH2+]2', 'Clc1cc(Cl)nnc1N/C=C/c1ccc(CCCO)o1', 'Clc1ccc(-c2nnnn2CCC[NH+]2CCCCCC2)nc1', 'Clc1ccc(-c2nnnn2CC(=O)N2Cc3ccccc32)cc1', 'Clc1ccc(-c2nnnn2CCc2cnc3ccccc3n2)cc1', 'Clc1ccc(-c2nnnn2CC2CC2)cc1F', 'Clc1ccc(-c2nnnn2CCNc2ccc(Cl)cc2)cc1', 'Clc1ccc(-c2nnnn2CCSCc2ccc(Cl)o2)cc1', 'Clc1ccc(-c2nnnn2CC[NH+]2CCCC2)cc1', 'Clc1ccc(-c2nnnn2CC[NH2+]C2CC2)cc1', 'Clc1ccc(-c2cc[nH]n2)o1\n\n\n\n', 'Clc1ccc([C@@H]([O-])NCc2cc(Cl)ccc2[N+](=O)[O-])cn1', 'Clc1ccc([C@@H]([O-])N(C[C@H]2CCOC2)C2CC2)cc1', 'Clc1ccc([C@@H]([O-])Nc2ccc(Br)cc2Cl)cc1', 'Clc1ccc([C@@H]([O-])N2C[C@@H](CNc3ncccc3C#N)CC2)cc1', 'Clc1ccc([C@@H]([O-])N[C@@H](CC(F)(F)F)c2ccccc2)cc1', 'Clc1ccc([C@@H]([O-])N/N=C/c2ccco2)cc1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1/C=C/CC[NH2+]C(CC(=O)[O-])c1ccccc1', 'Clc1ccsc1/C=[NH+]/C[NH+]1CCC2(CCOCC2)C1', 'Clc1cc(Br)cc2cccnc12', 'Clc1cc(Br)cc2c1OCCN1C[C@@H]1C[C@@H]2C(=O)[O-]', 'Clc1cc(Br)cc2cnccc12', 'Clc1cccc([S@@]c2n[nH]c(=O)n2C[C@H]2CCCO2)c1', 'Clc1cccc([S@@]c2nc3ccccc3c(=O)n2C[C@H]2OCCO2)c1', 'Clc1cccc([S@@]c2nnc(-c3ccccc3Br)n2C2CC2)c1', 'Clc1cccc(S(=O)(=S)Nc2ccccc2C(F)(F)F)c1', 'Clc1cccc([C@@H](NCc2cc3nc[nH]c3c(Cl)c2Cl)C2CC2)c1', 'Clc1cccc([C@@H](N[C@H]2CC[NH+](Cc3ccco3)c3ccccc3)C2)c1', 'Clc1cccc([C@@H](Nc2nccc(C(N)=O)c2)C2CC2)c1', 'Clc1cccc([C@@H](N[C@@H]2CCc3nc[nH]c3C[NH+]2CC(F)(F)F)c2cccnc2)c1', 'Clc1ccc(Br)cc1-c1cnc2ccc(F)c(Br)c2c1N', 'Clc1ccc(Br)cc1-c1cnc2n1CCN(Cc1cccc([N+](=O)[O-])c1)C2', 'Clc1ccc(Br)cc1-c1ccnc(N2CCN(Cc3ccccc3Cl)CC2)c1', 'Clc1ccc(Br)cc1-c1ccncc1', 'Clc1ccc(Br)cc1-c1cnn(-c2ccc(Cl)cc2Cl)c1', 'Clc1ccc(Br)cc1-c1cnn(CCN2CCCC2)c1', 'Clc1ccc(Br)cc1-c1cnn([C@H]2CC[NH+](C3CCOCC3)C2)n1', 'Clc1ccc(Br)cc1-c1cnn([C@@H]2CCC[NH2+]C2)n1', 'Clc1ccc(Cc2csnn2)cc1', 'Clc1ccc(Cc2csnn2)cc1NCCO', 'Clc1ccc(Cc2c[nH]nc2C[NH+]2CCCC[C@@H]2c2cccnc2)cc1', 'Clc1cc(C(=O)COC[C@H]2COc3ccccc3O2)c(F)cc1F', 'Clc1cc(CNCC[NH+]2CCC[C@H]2C[NH+]2CCCC2)cnc1C1CC1', 'Clc1cc(CNC(=O)c2cc(C3CCCCC3)no2)cc([N+](=O)[O-])c1O', 'Clc1cc(CNCc2ccc(C3CC3)cc2)n[nH]1', 'Clc1cc(CNC2CC2)nc(-c2ccc(F)c(F)c2)n1', 'Clc1cc(CNC[C@@H](c2ccc3c(c2)OCO3)c2ccccc2)ncn1', 'Clc1cc(CNC[C@H](N)c2ccccc2)[nH]n1', 'Clc1cc(CNC[C@](O)(c2ccccc2)c2ccc(C#N)cc2)cc([O-])n1', 'Clc1ccc(Br)cc1Cc1ccnc(F)c1', 'Clc1ccc(Br)cc1Cc1nnc(-c2ccc(Cl)cn2)n1N', 'Clc1ccc(Br)cc1S[C@H]1CCCc2ccccc12', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O[C@H]1CC[C@@H]([NH+]2CCCC2)C1', 'Clc1ccc(-c2n[nH]cc2Br)cc1', 'Clc1ccc(-c2n[nH]cc2NC(=O)c2cc([N+](=O)[O-])ccc2Cl)cc1', 'Clc1ccc(-c2n[nH]cc2N2CCN(c3ncccn3)CC2)cc1', 'Clc1ccc(-c2n[nH]cc2Br)cc1', 'Clc1ccc(-c2n[nH]cc2Br)nc1', 'Clc1ccc(-c2nonc2Cc2cccc(Br)c2)cc1', 'Clc1ccc(-c2nonc2[S-])s1', 'Clc1ccc(C[NH2+]c2ccnc(-c3ccco3)n2)cc1', 'Clc1ccc(C[NH2+]c2nc(C3CC3)n(C3CCCC3)n2)cc1', 'Clc1cc(/C=C(/C#CCN2CCOCC2)c2ccccc2)ccc1Br', 'Clc1cc(/C=C(/NC2=NN=CC2)Sc2ccccc2)[n+]([O-])c2ccccc12', 'Clc1cc(/C=C(/[O-])/N=[N+]=NC(=O)c2ccccc2)cc(Br)c1O', 'Clc1cc(/C=C(/Br)C(F)(F)F)ccc1Cl', 'Clc1ccccc1Oc1cncc(-c2ccccc2)n1', 'Clc1ccccc1Oc1cncc2ccccc12', 'Clc1ccccc1Oc1cnccc1Oc1ccc(OC(F)(F)F)cc1', 'Clc1ccccc1Oc1cnccn1', 'Clc1ccccc1Oc1cc([N+](=O)[O-])cc2c1OCOC2', 'Clc1ccccc1Oc1cc(NCc2ccccc2Cl)ncn1', 'Clc1ccccc1Oc1cc(Nc2ncno2)ccc1F', 'Clc1ccccc1Oc1cc(N2CCOCC2)ccc1O', 'Clc1ccccc1Oc1cnc2nc(-c3ccc(Br)o3)[nH]c2c1', 'Clc1ccccc1Oc1cnc2n1C[C@@H](OCc1cccnc1)C2', 'Clc1ccccc1Oc1cnc2nnc(-c3ccncc3)n2c1', 'Clc1ccccc1OCC(=O)Nc1ccc(C(=O)[O-])cc1', 'Clc1ccccc1OCC(F)(F)F', 'Clc1ccccc1OCc1cc(F)ccc1F', 'Clc1ccccc1OCc1cccc(COCCO)n1', 'Clc1ccccc1OCc1ccncc1', 'Clc1ccccc1OCCOCCOC[C@@H]1CCCO1', 'Clc1ccccc1OCCO[C@H]1C[C@@H]1C1CCCC1', 'Clc1ccccc1OCCOc1ncccc1F', 'Clc1ccccc1Oc1cc(CCCn2cccn2)ncn1', 'Clc1ccccc1Oc1cc(C(F)(F)F)ccc1F', 'Clc1ccccc1Oc1cc(Cc2csc(-c3ccco3)n2)cnc1[O-]', 'Clc1ccccc1Oc1cc(COc2ccc(Br)cc2)ccn1', 'Clc1ccccc1Oc1cc(C[NH+]2CCC(Cc3nc4ccccc4[nH]3)CCCC2)ccc1O', 'Clc1ccccc1Oc1cc(C[NH2+][C@@H]2CCOC3(CCCCC3)C2)ncc1SC', 'Clc1ccccc1Nc1cc(OC[C@H]2CCCO2)ncn1', 'Clc1ccccc1Nc1cccc(Cn2ccc3ccccc32)c1', 'Clc1ccccc1Nc1ccnc2ccccc12', 'Clc1ccccc1Nc1nn[n-]c1C(=O)[O-]', 'Clc1ccccc1Nc1nncc(-c2ccccc2Br)n1', 'Clc1ccccc1Nc1nnnn1-c1ccccc1', 'Clc1ccccc1-c1cnnn1[C@@H]1CC[C@](O)(C(F)(F)F)C1', 'Clc1ccccc1-c1cnnc(O[C@@H]2CCOC3(CCCC3)C2)n1', 'Clc1ccccc1-c1cnn2c1NCCC[C@@H]2[NH2+]CCC(C)C', 'Clc1ccccc1NS(=O)(=O)c1ccc(C(=O)[O-])cc1', 'Clc1ccccc1NS(=O)(=O)c1ccccc1Br', 'Clc1ccccc1NS(=O)(=O)c1c[nH]c2nccc(Cl)c12', 'Clc1ccccc1NS(=O)(=O)c1cccc(C(F)(F)F)c1', 'Clc1ccccc1NS(=O)(=O)c1cc(Cl)cc(Cl)c1', 'Clc1ccccc1NS(=O)(=O)c1ccccc1N[C@H]1CCSc2ccccc21', 'Clc1ccccc1NS(=O)(=O)c1cc2c(cc1Cl)OCCO2', 'Clc1ccccc1Nc1cn(-c2ccccc2)nc1-c1ccccc1', 'Clc1ccccc1Nc1cn([C@H]2CCOC3(CCOCC3)C2)[nH]c1=O', 'Clc1ccccc1NS[C@H]1Oc2cc(Cl)ccc2S1', 'Clc1ccccc1NS[C@H]1CCCN(CCc2cccnc2)C1', 'Clc1ccccc1NS[C@H]1NC(=O)c2cc(C(=O)NC3CCCCCC3)ccc21', 'Clc1ccccc1NS[C@H]1[NH+](C2CC2)CCC[NH+](C2CC2)[C@@H]1C', 'Clc1cccc(S(=O)(=O)Nc2ccc3c(c2)OCCO3)c1O', 'Clc1cccc(F)c1-c1cccc(C(F)(F)F)c1', 'Clc1cccc(F)c1-n1cc(Cl)ccc1=O', 'Clc1cccc(S[C@@H]2C[NH2+]C[C@H]2CO)c1', 'Clc1cccc(S[C@@H]2OCCNC2=O)c1', 'Clc1cc(F)c(I)cc1I', 'Clc1cc(F)c(Cl)cc1N[C@@H]1CCOc2ccccc21', 'Clc1cc(F)c(Cl)nc1NC[C@@H]1CCCN1C(F)(F)F', 'Clc1ccc(-c2n[nH]c(NCc3cccc([N+](=O)[O-])c3)n2)cc1', 'Clc1ccc(-c2n[nH]c(Nc3ncccn3)n2)cc1', 'Clc1ccc(-c2n[nH]c(NNc3ccccc3Cl)n2)cc1', 'Clc1ccc(-c2n[nH]c(N/N=C(\\[O-])c3ncns3)n2)cc1', 'Clc1ccc(-c2cc(Br)ccc2ONCCCO)c(C(=O)[O-])c1', 'Clc1ccc(-c2cc(Br)co2)cc1F', 'Clc1ccc(-c2cc(Br)c3c(c2)OCCO3)cc1', 'Clc1ccc(-c2cc(Br)cnc2Cn2ccnc2)cc1', 'Clc1ccc(-c2cc(Br)cs2)cc1', 'Clc1ccc(-c2cc(Br)c[nH]c2=O)cc1', 'Clc1ccc(-c2cc[nH]n2)o1\n\n\n\n\n', 'Clc1ccc(-c2cc(F)c(C[NH2+]Cc3cccs3)cc2)cc1', 'Clc1ccc(-c2cc(F)c(=O)[nH]c(=O)n2)cc1', 'Clc1ccc(-c2cc(F)c(O)cc2)c(F)c1', 'Clc1ccc(-c2cc(F)c(F)c(F)c2COc2ccc(CC(=O)[O-])cc2)cc1', 'Clc1ccc(-c2cc(F)c([C@@H]3CCCN(c4nccs4)c3)CCN2)cc1', 'Clc1ccc(-c2cc(F)c(Cl)cc2O)cc1', 'Clc1ccc(-c2cc(F)c(/C=C/C(=O)[O-])o2)cc1', 'Clc1cc(N2CCc3c(c(OCc4ccccc4)nn3Cc3cccc(F)c3)C2)ccc1[N+](=O)[O-]', 'Clc1cc(N2CCc3nc(CSc4ncccc4O)[nH]c3N2)cs1', 'Clc1cc(N2CCc3sccc3C2)ccc1C(F)(F)F', 'Clc1ccccc1CI\n\n\n\n', 'Clc1ccccc1C1CCN(S(=O)(=O)c2cc(OC(F)(F)F)ccc2Cl)CC1', 'Clc1ccccc1CN(Cc1ccccc1)Cc1ccccn1', 'Clc1ccccc1CN(c1ccccc1)S(=O)(=O)c1ccc(Cl)cc1', 'Clc1ccccc1CI\n\n\n\n\n', 'Clc1ccccc1C1(C[NH2+]Cc2ccccc2)CCCC1', 'Clc1ccccc1C1(Oc2ccccc2)CC1', 'Clc1ccccc1C1([O-])CC1', 'Clc1ccccc1CI\n\n\n\n\n\n', 'Clc1ccccc1CNc1cnn(-c2cc3ccccc3nc2Cl)c1', 'Clc1ccccc1CI\n\n\n\n\n\n\n', 'Clc1ccccc1OCc1ccn[nH]1', 'Clc1ccccc1OCc1ccn(Cc2ccccc2Cl)n1', 'Clc1ccccc1OCc1ccncc1COCC1CC[NH+](C)CC1', 'Clc1ccccc1OCc1ccnn1C1CC1', 'Clc1ccccc1OC1C=C[C@H](OC2CCCCC2)[NH+]1', 'Clc1ccccc1OC1CCC([NH3+])CC1', 'Clc1ccccc1OC1CN(C(=O)c2cc(OCc3ccco3)on2)C1', 'Clc1ccccc1OC1C[NH2+]CC1', 'Clc1ccccc1Cc1noc(Cc2ccc(Br)s2)n1', 'Clc1ccccc1Cc1n[nH]c(Nc2cc(Cl)ccc2Cl)n1', 'Clc1ccccc1Cc1nc(Cc2[nH+]cc(Cl)cc2Cl)n[nH]1', 'Clc1ccccc1Cc1nnc([C@@H]2CCCC[NH+]2Cc2cccc(C(F)(F)F)c2)o1', 'Clc1cc(Br)c2nc(O)sc2c1', 'Clc1cc(Br)c2nc(O)ncc2c1O', 'Clc1ccc(C[NH+](CC(=O)N2CCC[C@@H]2CC2CCCCC2)C2CC2)cc1', 'Clc1ccc(C[NH+](CCc2ccco2)[C@@H]2CCCc3cc(Cl)ccc32)cc1F', 'Clc1ccc(C[NH+](CC2CC2)C2CC2)cc1', 'Clc1ccc(C[NH+](CCO)[C@@H]2CCC[C@H]2S(C)(=O)=O)cc1Cl', 'Clc1ccc(C[NH+](CC[C@@H]2CCCO2)Cn2cccn2)nc1', 'Clc1ccc(C[NH+](CC[C@H]2CCCCO2)c2ncccc2C(=O)[O-])cc1', 'Clc1cc(Br)cn1[C@@H]1CCO[C@]2(CCOC2)C1', 'Clc1cccc([S@@]C2SCCCC2)c1', 'Clc1cccc([S@@]C2=NC(=O)[C@@H]3CCC(=O)N32)c1', 'Clc1cccc([S@@]C2CCN(S(=O)(=O)c3cccnc3)CC2)c1', 'Clc1cccc([S@@]C2(COC(=O)Nc3ccc(-n4cccn4)cc3)CCCC2)c1', 'Clc1ccccc1[C@H]([NH3+])c1csc(Br)c1', 'Clc1cc(C2CC[NH+](C[C@H](c3cccnc3)c3cccs3)CC2)ncn1', 'Clc1cccc(F)c1Br', 'Clc1cccc(-c2n[nH]/c(=C/c3ncccc3Cl)o2)c1', 'Clc1cccc(-c2n[nH]cc2C2=NN=C(Sc3nccn3Cc3ccccc3)C2)c1', 'Clc1cccc(F)c1[C@@H](O)c1cccc(OCc2ccccc2)c1', 'Clc1cccc(F)c1[C@@H]1Oc2c(cccc2Cl)C1', 'Clc1cccc(F)c1Br\n', 'Clc1cccc(F)c1CCNC[C@@H]1CCc2ccccc21', 'Clc1cccc(F)c1C(=O)Nc1ccc(-c2cscn2)cc1', 'Clc1cccc(F)c1Cc1nc(CCCc2ccccc2)no1', 'Clc1cccc(F)c1C1CC[NH+]([C@@H]2CCN(c3ccccc3)C2)CC1', 'Clc1cccc(F)c1COc1cccc(Cl)c1OCCC1CC1', 'Clc1cccc(F)c1CNCc1ccccc1F', 'Clc1cccc(F)c1C[C@@H](C[NH3+])c1c(F)cccc1F', 'Clc1cccc(F)c1Cn1ccc(-c2ccc(Cl)cc2)n1', 'Clc1cccc(F)c1CSc1cccc(Cl)c1', 'Clc1cccc(F)c1C[NH+]1CCC[C@H]1C[C@@H]1CCC(=O)NC2=C1CCCC2', 'Clc1cccc(/N=N\\NC(=S)NNC(=O)Cc2ccccc2)c1', 'Clc1cccc(/N=N/[C@H]2c3cc(OCc4ccccc4)ccc3C2=O)c1', 'Clc1cccc(-c2cccc(OCc3ccccn3)c2)c1', 'Clc1cccc(-c2ccoc2Br)c1', 'Clc1cccc(-c2ccnc(Sc3cccc(C(F)(F)F)c3)n2)c1', 'Clc1cccc(-c2ncc(CSCc3n[nH]c(-c4ccccc4)n3)n2Cn2cncn2)c1', 'Clc1cccc(-c2ncoc2C2CCN(c3ccc(N4CCOCC4)cc3)CC2)n1', 'Clc1cccc(-c2nc3cc(Cl)ccc3o2)c1', 'Clc1cccc(-c2ncnn2Cc2ccco2)c1', 'Clc1cccc(C[NH2+]Cc2nc(C3CC3)ns2)c1', 'Clc1cccc(C[NH2+][C@H]2CC[C@@H](S)C2)c1', 'Clc1cccc(C[NH2+][C@@H]2CCCC[C@@H]2c2cccc(F)c2)c1', 'Clc1ccccc1Oc1cncc2ncnc2c1O', 'Clc1cccc(Br)c1NC1CC[NH+](CCN2CCCC2)CC1', 'Clc1cccc(Br)c1N1CC[NH+](Cc2cccnc2Cl)CC1', 'Clc1cccc(Br)c1N[C@@H]1CCOC2(C[C@@H](COCc3ccccc3)C2)C1', 'Clc1ccc(Br)cc1C1=c2ccccc2=[NH+]C1', 'Clc1ccc(Br)cc1C1CC[NH+](Cc2ccc(CO)o2)CC1', 'Clc1ccccc1-c1nccs1', 'Clc1ccccc1-c1ncc[nH]c1=O', 'Clc1ccccc1-c1ncc[nH]1', 'Clc1ccccc1-c1nc(-c2cccs2)no1', 'Clc1ccccc1-c1noc(Cc2ccccc2)n1', 'Clc1ccccc1-c1noc(Cc2ccccc2F)n1', 'Clc1ccccc1-c1noc([C@@H]2CN(Cn3cccc3F)C2)n1', 'Clc1ccccc1-c1noc([C@H]2CCCN(C(=O)c3ccccc3)C2)c1Cl', 'Clc1ccccc1-c1noc(/C=C/c2ccc(F)cc2Cl)n1', 'Clc1ccccc1-c1n[nH]cc1C[NH+]1CCC(C2CCN(C(=O)c3ccco3)CC2)C1', 'Clc1ccccc1[C@H](Cl)C1CCCC1', 'Clc1ccccc1[C@H](Cl)c1ccccc1OCc1ccccc1Cl', 'Clc1ccccc1[C@H](Cl)[C@@H]1CN(C2CC2)CCO1', 'Clc1ccccc1[C@H](Cl)[C@H]1COc2ccccc21', 'Clc1ccccc1[C@H](Cl)[C@](Cc1ccccc1)[NH+]1CCCC1', 'Clc1ccccc1[C@H](OCc1ccco1)C1CC1', 'Clc1ccccc1[C@H](O)[C@@H]1CCO[C@]2(CCSC2)C1', 'Clc1ccccc1O/Cn1c(=S)oc2ccc(Cl)cc21', 'Clc1ccccc1O/Cn1c2ccccc2[nH+]2ncnc21', 'Clc1ccccc1O/Cn1cnn[nH+]1', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc1n1/C(=C/C(=O)N(Cc3ccccc3)c3ccccn3)c([O-])c12', 'Clc1cccc2cc[nH]c12', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c12', 'Clc1cccc2c1NC(=O)N2[C@H](C(=O)N(Cc1cccc(F)c1)c1ccccc1)c1ccccc1', 'Clc1cccc2c1N(Cc1nccn1Cc1ccccc1Cl)CCCC2', 'Clc1cccc2c1Nc1nnc(SCc3ccc(Br)s3)nc12', 'Clc1cccc2c1N[C@@H](c1ccc([N+](=O)[O-])cc1)[C@@H]1CC=C[C@@H]21', 'Clc1cccc2c1N[C@H](c1ccc(NCc3ccc(Cl)cc3)[nH]c1=O)CC(O)(CO)C2', 'Clc1ccsc1N/C(=C/c1ccc(Cl)cc1)NCc1ccccc1', 'Clc1ccsc1N/C1=C2C=C[C@H](C2)c2ccccc21', 'Clc1cc(F)cc2nc[nH]c12', 'Clc1cc(F)cc2nccn12', 'Clc1cc(F)cc2ncn(CCC[NH+]3CCCCCC3)c12', 'Clc1ccc(Br)cc1S[C@H]1CCOCC1', 'Clc1ccc(Br)cc1S[C@H]1Cc2ccccc2C1', 'Clc1ccc(Br)cc1S[C@H]1C[C@@H](O)CN1C(=O)C1CC1', 'Clc1ccc(-c2nn(-c3ccc([N+](=O)[O-])cc3)c(Cl)c2Cl)cc1', 'Clc1ccc(-c2nn(-c3ccc(OCc4ccccc4)cc3)c(Cl)c2)cc1', 'Clc1ccc(-c2nn(-c3ccc(F)cc3)c3c2NCCC[C@H]3OC(F)(F)F)cc1', 'Clc1ccc(-c2nn(-c3ccc(-c4cccs4)cc3)nn2)cc1', 'Clc1ccc(-c2nn(-c3ccc(S(=O)(=O)C4CCCCC4)cc3)o2)cc1', 'Clc1ccc(-c2nn(-c3ccc(Cl)cc3Cl)cn2)cc1', 'Clc1ccc(-c2nn(-c3ccc(Br)cc3)c3ccccc23)cc1', 'Clc1ccc(-c2n[nH]cc2-c2ccc(S(=O)(=O)C(F)(F)F)cc2)cc1', 'Clc1ccc(-c2n[nH]cc2N2CCCC2)nc1[N+](=O)[O-]', 'Clc1cccc([S@@]c2c[nH]c3ccccc32)c1', 'Clc1cccc([S@@]c2c[nH]nc2C[NH+](CC2CCCC2)C2CC2)c1', 'Clc1ccc(-c2nonc2N2CCCCC2)cc1', 'Clc1ccc(-c2nonc2N2CC(CO)C2)cc1', 'Clc1ccc(-c2nonc2N2CCc3ccccc3CC2)cc1', 'Clc1ccc(-c2nonc2N2CCOCC2)cc1', 'Clc1ccc(-c2nonc2N2CCN(Cc3cc(Cl)cs3)CC2)cc1', 'Clc1cc(NC[C@@H](c2ccccc2)n2cc(S(=O)(=O)N3CCCCC3)cc2)ccc1Cl', 'Clc1cc(NC[C@@H]2CCS(=O)(=O)C2)ncn1', 'Clc1ccccc1-c1ncc(-c2ccccc2Cl)nc1N1CCCC1', 'Clc1ccccc1-c1ncc(COc2ccccc2)[nH]1', 'Clc1ccccc1-c1ncc(S(=O)(=O)N2CCC[C@H]2C(N)=O)cn1', 'Clc1ccccc1-c1ncc(F)cc1C[S@@](=O)c1cscc1C(F)(F)F', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n', 'Clc1ccccc1Cn1cccc(Br)c1=O', 'Clc1ccccc1Cn1nnc(-c2cccc(F)c2)n1', 'Clc1ccccc1CO[C@H](c1ccco1)c1ccccc1', 'Clc1ccccc1CO[C@H]1CCC[C@H]1Cc1ccc(C(F)(F)F)cc1', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1C[NH+]1CCN(c2c(Cl)cccc2[N+](=O)[O-])CC1', 'Clc1ccccc1C1=CCN(c2ccc(F)cc2)CC1', 'Clc1ccccc1Cn1c[nH+]c(-c2cccc(Cl)c2)n1', 'Clc1ccccc1Cn1c(SCc2ccccc2)nc2ccccc21', 'Clc1ccccc1Cn1cc([N+](=O)[O-])cn1', 'Clc1ccccc1Cn1c2ccccc2c2ncnn21', 'Clc1ccccc1Cn1cnc2ccccc21', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1C1CCN(CC(=O)Nc2ccccc2F)CC1', 'Clc1ccccc1C1CC[C@H](c2ccccc2F)C1', 'Clc1ccccc1C1CC1', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1COCCSc1ccccc1', 'Clc1ccccc1COC(=O)c1ccc2c(c1)N(Cc1cc(F)cc(F)c1F)CC2', 'Clc1ccccc1COCc1cccc(COc2ccccc2)c1', 'Clc1ccccc1COC1CCC(CNC(=O)CCOc2ccccc2)CC1', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1C[NH+]1CCc2ccccc2[C@H]1C(=O)Nc1cccc(Cl)c1', 'Clc1ccccc1C[NH+]1C[C@@H]2CCC[C@@H](O)[C@@H]2C1', 'Clc1ccccc1-c1cscc1F\n', 'Clc1ccccc1-c1cscc1-n1ccnc1', 'Clc1ccccc1-c1cscc1F\n\n', 'Clc1ccccc1-c1cscc1-c1ccccc1C(=O)N[C@@H]1CCCc2ccccc21', 'Clc1ccccc1-c1cscc1F\n\n\n', 'Clc1ccccc1-c1csc(-c2ccco2)n1', 'Clc1ccccc1-c1cscc1-c1ccc(Br)cc1', 'Clc1ccccc1-c1csc(Cl)n1', 'Clc1ccccc1-c1cscc1F\n\n\n\n', 'Clc1ccccc1-c1csc(/C=C\\c2cccc(O)c2)n1', 'Clc1ccccc1-c1csc(/N=C2\\CCCN2CC(F)(F)F)n1', 'Clc1ccccc1-c1cscc1-n1cccn1', 'Clc1ccccc1-c1cscc1-n1nccc1SCc1ccccc1F', 'Clc1ccccc1-c1csc(SC[C@H](O)C2CC2)n1', 'Clc1ccccc1-c1csc(S(=O)(=O)Cc2ccccc2)n1', 'Clc1ccccc1-c1csc(Sc2cccnc2)n1', 'Clc1ccccc1NCc1ccc(Cl)o1', 'Clc1ccccc1NCc1nnc2n1CCCCC2', 'Clc1ccccc1-c1noc(CCCS(=O)(=O)c2ccccc2)n1', 'Clc1ccccc1-c1noc(C(=O)N2CCC(Cc3ccco3)CC2)n1', 'Clc1ccccc1-c1noc(Cc2cccs2)n1', 'Clc1ccccc1-c1noc(C2CCCCC2)n1', 'Clc1ccccc1-c1noc(CN2CC[NH2+]CC2)n1', 'Clc1ccccc1-c1noc(CSc2nnc(C3CC3)s2)n1', 'Clc1ccccc1-c1noc(C[NH+]2[C@@H](C(=O)[O-])CCCN2)n1', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CNCCc1ccn(-c2ccccc2)n1', 'Clc1ccccc1CNC(=O)N1CCCc2ccccc21', 'Clc1ccccc1CNCc1ccc2c(c1)OCO2', 'Clc1ccccc1CNC1CCN(CN2CCc3ccccc32)CC1', 'Clc1ccccc1CNC[C@@H](O)COc1ccccc1', 'Clc1ccccc1CNC[C@H](O)c1ccccc1', 'Clc1ccccc1C1=CCN=CN1', 'Clc1ccccc1C1=C(c2ccc(OCn3cccn3)cc2)Nc2ccccc21', 'Clc1ccccc1C1=CO[C@H]2C[C@@H](CC(=O)OCc3cccc(Cl)c3)N12', 'Clc1ccccc1C1=CN(c2ccc3nnnn3n2)CC1', 'Clc1ccccc1C1=C[C@@H]2[C@@H](CCCc3ccccc31)[C@@H]1CCC[C@@H]12', 'Clc1ccccc1C1=CSCC[NH2+]CC1', 'Clc1ccccc1[C@H]([NH2+]C1CCCCC1)[C@H]1CO[C@@H]2C[C@@H]1c1ccccc1C2', 'Clc1ccccc1[C@H]([NH2+]/N=C/c1cccc(Cl)c1)n1cnnn1', 'Clc1ccccc1[C@H]([NH2+][C@H]1CCc2cccnc21)c1ccccc1', 'Clc1ccccc1[C@H]([NH2+][C@@H]1CC[C@@]1(O)COc1ccccc1)c1ccc(Br)o1', 'Clc1cccc(/N=N\\Cc2ccc[nH]2)c1F', 'Clc1cccc(/N=N\\Nc2ncccn2)c1', 'Clc1cccc(/N=N\\c2cccc(Br)c2)c1', 'Clc1cccc(NC2CCN(C(=O)C3=c4ccccc4=[NH+]C3)CC2)c1', 'Clc1cccc(NC2(c3cccs3)CC2)c1F', 'Clc1cccc(NC2=[NH+]C[C@@H]3CCCC[C@@H]3C2)c1', 'Clc1cccc(NC2[C@@H](N3CCC(c4ccccc4)CC3)C2)c1', 'Clc1cccc(NC2[C@H](Cc3ccccc3)O2)c1', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc1c1nc2cc(Cl)ccc1OCC1CC1', 'Clc1ccccc1Oc1cnc2sc(-c3ccc(Br)cc3)nc2c1', 'Clc1ccccc1Oc1cnc2snc(-c3ccc(Br)cc3)c2c1', 'Clc1ccccc1OCCNC(=O)[C@@H](c1ccco1)C(F)(F)F', 'Clc1ccccc1OCCN(C(=O)Nc1cccc2[nH]ccc12)C1CC1', 'Clc1ccccc1OCCNc1ncccn1', 'Clc1ccccc1OCCN1CC[NH+](CC2CC2)CC1', 'Clc1ccccc1Oc1cnc2cc(F)ccc2n1', 'Clc1ccccc1Oc1cncc(Cn2ccc([N+](=O)[O-])n2)c1', 'Clc1ccccc1Oc1cncc(Oc2ccc(Cl)cc2)n1', 'Clc1ccccc1Oc1cncc(N2CCC[C@H](Oc3ccccc3F)C2)n1', 'Clc1ccccc1Oc1cncc(-c2ccc(F)cc2Cl)n1', 'Clc1ccccc1Oc1cncc(Cl)n1', 'Clc1ccccc1Oc1cncc([O-])n1', 'Clc1ccccc1Oc1cncc(Br)c1', 'Clc1ccccc1Oc1cncc(/C=C/N2CCN(CCC[NH+]3CCCCCC3)CC2)c1', 'Clc1ccccc1Nc1ccc(F)cc1F', 'Clc1ccccc1Nc1ccccc1Cl', 'Clc1ccccc1Nc1ccc2nnc(-c3c(F)cccc3Cl)n2c1', 'Clc1ccccc1N1Nc2[nH+]ccnc2NC1', 'Clc1ccccc1N1Nc2ccccc2NC1=O', 'Clc1ccccc1N1Nc2ncccc2C1', 'Clc1ccccc1-c1cscc1F\n\n\n\n\n', 'Clc1ccccc1-c1cscc1C(=O)N1CCc2c([nH]c3ccccc23)C1', 'Clc1ccccc1-c1cscc1C(F)(F)F', 'Clc1ccccc1-c1cscc1F\n\n\n\n\n\n', 'Clc1ccccc1-c1cscc1-c1c[nH]c2ccccc12', 'Clc1ccccc1-c1cscc1-c1cc2c(cc1Cl)OCCO2', 'Clc1ccccc1-c1csc([C@@H](OCC(F)(F)F)c2cc(F)cc(F)c2F)n1', 'Clc1ccccc1-c1csc([C@@H]2CCCN(S(=O)(=O)CCN3CCOCC3)C2)[nH+]1', 'Clc1ccccc1OCc1csc(-c2ccco2)n1', 'Clc1ccccc1OCc1cscc1Br', 'Clc1ccccc1OCc1csc2ccccc12', 'Clc1ccccc1OCc1csc2[nH]nc(-c3cccnc3)c12', 'Clc1ccccc1OCc1csc2ccc(F)cc12', 'Clc1ccccc1OCc1csc2ncc(-c3ccccc3Cl)n12', 'Clc1ccccc1OCCn1c(=O)[nH]c2ccccc21', 'Clc1ccccc1OCCCCn1cc(Br)cn1', 'Clc1ccccc1OCCC(=O)N1CCC[C@H](C(=O)N2CCCC2)C1', 'Clc1ccccc1OCCCc1nnc2n1CCC2', 'Clc1ccccc1OCCC1CCCCC1', 'Clc1ccccc1OCCCOc1ccc(Cl)cc1', 'Clc1ccccc1OCCCNCc1csnn1', 'Clc1ccccc1OCCCn1ccc2ccccc21', 'Clc1ccccc1OCCCSc1nnc2n1CCCC2', 'Clc1ccccc1OCCC[NH+]1CCCCC1', 'Clc1ccccc1OCCC[NH2+]Cc1ccc(OC(F)F)cc1', 'Clc1ccccc1OCc1cc([N+](=O)[O-])ccc1Br', 'Clc1ccccc1OCc1cc(COC(F)(F)F)cs1', 'Clc1ccccc1OCc1cc(=O)n2nc(Br)sc2n1', 'Clc1ccccc1OCc1cc(OCc2ccc(F)cc2)nc2ccccc12', 'Clc1ccccc1OCc1cc(F)cc2ccc(F)cc12', 'Clc1ccccc1OCc1cc(-c2ccccc2O)nc2ccccc12', 'Clc1ccccc1OCc1cc(Cl)ccc1F', 'Clc1ccccc1OCc1cc(Br)ccc1Br', 'Clc1ccccc1Oc1cc(Br)ccc1O', 'Clc1ccccc1Oc1cc(-c2cccs2)n[nH]1', 'Clc1ccccc1OCCS(=O)(=O)C1CC1', 'Clc1ccccc1OCCS[C@H]1[C@@H]2CCC[C@@H]2C1', 'Clc1ccccc1OCCSc1ccccc1F', 'Clc1ccccc1Oc1cc(Cl)ccc1C(=O)NC[C@@H]1CCOC1', 'Clc1ccccc1Oc1cc(F)ccc1O', 'Clc1ccccc1Oc1cc(Br)ccc1F', 'Clc1ccccc1Oc1cc([N+](=O)[O-])c2ccccc2n1', 'Clc1ccccc1O/N=C/c1ccccc1O', 'Clc1ccccc1O/N=C(\\[O-])CCCn1nc2ccccn2c1=O', 'Clc1ccccc1O/N=C\\c1cccc(Cl)c1', 'Clc1ccccc1O/N=C1\\SC(=C(Cl)Cl)N1c1cccc(F)c1', 'Clc1ccccc1OC1(Cl)CCOCC1', 'Clc1ccccc1Oc1cc(-c2ccccc2)nc2ccc(F)cc12', 'Clc1ccccc1Oc1cc(F)c(OCc2nc(-c3ccc(Cl)cc3)[nH]n2)cc1', 'Clc1ccccc1Oc1cc(F)ccc1F', 'Clc1ccccc1Oc1cc(Cl)c(Br)cc1F', 'Clc1ccccc1Oc1cc(Cl)ccc1Cl', 'Clc1ccccc1Oc1cc(Cl)c2[nH]nnc2c1', 'Clc1ccccc1OCc1csc(C(=O)[O-])n1', 'Clc1ccccc1OCc1csc(OC2CC2)n1', 'Clc1ccccc1OCc1csc(N2CCCCCCCC2)n1', 'Clc1ccccc1OCc1csc(-c2ccc(Cl)cc2)n1', 'Clc1ccccc1OCc1csc(Cl)c1', 'Clc1ccccc1OCc1csc([C@H]2CCC[NH2+]C2)n1', 'Clc1ccccc1OCc1csc(Br)c1', 'Clc1ccccc1Nc1nnnn1-c1ccccc1F', 'Clc1ccccc1Nc1nnc(Cc2ccccn2)o1', 'Clc1ccccc1Nc1nncc([O-])c1[N+](=O)[O-]', 'Clc1ccccc1Nc1nnc2c(Br)ncncc12', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1C(=O)[C@@H]1CCCC[C@H]1O', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CO[C@H](CO)c1ccc(Cl)cc1', 'Clc1ccccc1CO[C@H](c1ccccc1)c1ccccc1', 'Clc1ccccc1CO[C@H](O)c1cscc1[N+](=O)[O-]', 'Clc1ccccc1CO[C@H]([NH3+])Cn1ccccc1=O', 'Clc1ccccc1CO[C@H]([NH2+]Cc1ccc(F)cc1)c1ccccc1F', 'Clc1ccccc1Oc1cc(Br)c[nH]1', 'Clc1ccccc1Oc1cc(Br)c(F)c(F)c1F', 'Clc1ccccc1Oc1cc(Br)ccc1F', 'Clc1ccccc1Cn1nc2cccnc2c1CSCCO', 'Clc1ccccc1Cn1nnnc1[C@@H]1CNC(=O)N1C[C@H]1CC=CCC1', 'Clc1ccccc1Oc1cnc2cc(C(F)(F)F)c[nH]c12', 'Clc1ccccc1Oc1cnc2ccc(C3(c4ccccc4)c(=O)n3[C@H]3CCOC3)nc2c1', 'Clc1ccccc1NC(=O)[C@@H](CO)CCl', 'Clc1ccccc1NC(=S)Nc1nncs1', 'Clc1ccccc1NC(=S)Nc1cccc(Cl)c1', 'Clc1ccccc1NC(=O)/C=C/c1ccc([N+](=O)[O-])cc1', 'Clc1ccccc1NC(=S)NCc1ccc2c(c1)OCCO2', 'Clc1ccccc1NC(=S)c1ccc(CCC(=O)[O-])cc1', 'Clc1ccccc1N1NCC2(CCCCC2)O/C1=C/c1ccc(F)cc1', 'Clc1ccccc1N1NC(c2ccccc2)=NC1', 'Clc1ccccc1N1NC[C@@H](Cc2cnn(-c3ccccc3)n2)C1', 'Clc1ccccc1N1NC2(CCCCC2)N1', 'Clc1ccccc1O/N=C\\C(=C\\Br)c1ccc(Br)cc1', 'Clc1ccccc1O/N=C\\Nc1c(Cl)cccc1Cl', 'Clc1ccccc1O/N=C\\c1ccc(COc2ccccc2)cc1', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1C(=O)O[C@H]1C=NN=C[C@@H]1c1ccccc1', 'Clc1ccccc1C(=O)Nc1cnc2ccccc2c1O', 'Clc1ccccc1O/N=C/C(=O)Nc1ccc2[nH+]ccc(C3CC[NH+]3C)c2c1', 'Clc1ccccc1O/N=C/c1ccc(-c2ccccc2)o1', 'Clc1ccccc1O/N=C/Nc1c(Cl)cccc1[N+](=O)[O-]', 'Clc1ccccc1O/N=C/Sc1ccccc1[N+](=O)[O-]', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1Cc1cn(C[C@@H]2CCCC[C@H]2[NH3+])nn1', 'Clc1ccccc1Cc1cnn(CC(=O)N2CC[C@H](Cc3ccc(Cl)cc3)C2)c1', 'Clc1ccccc1Cc1cnc([C@@H]2CCO[C@H]2C(CC2CC2)c2ccccc2)c(Cl)c1', 'Clc1ccccc1Cc1cn2ccccc2n1', 'Clc1ccccc1-c1cscc1F\n\n\n\n\n\n\n', 'Clc1ccccc1-c1cscc1Cn1ccnc1C1CC1', 'Clc1ccccc1-c1cscc1F\n\n\n\n\n\n\n\n', 'Clc1ccccc1NCc1csc(N2CCCCC2)n1', 'Clc1ccccc1NCc1c[nH]nc1Cc1cccc(Br)c1', 'Clc1ccccc1NCc1ccc(Cl)c(Cl)c1', 'Clc1ccccc1NCc1cncn1Cc1cccs1', 'Clc1ccccc1Nc1cnc(Cl)nc1', 'Clc1ccccc1Nc1cnccc1C[NH2+][C@@H]1CCc2ccccc21', 'Clc1ccccc1NC(=O)CSc1nnnn1-c1ccccc1', 'Clc1ccccc1NC(=O)c1ccc2c(c1)N(Cc1ccccc1C(F)(F)F)CCO2', 'Clc1ccccc1NC(=O)N1CCN(c2ccccc2OC(F)F)CC1', 'Clc1ccccc1NC(=O)[C@@H]1CC(c2ccccc2)N=[N+]1c1ccccc1', 'Clc1ccccc1NC(=O)[C@H]1COc2ccccc2O1', 'Clc1ccccc1NC(=O)/C(=C/Cc1cccs1)c1ccccc1', 'Clc1ccccc1Nc1c2onc2ncn1Cc1ccccc1', 'Clc1ccccc1Nc1ccn(-c2ccc(Br)cc2)n1', 'Clc1ccccc1Nc1ccnc(Oc2ccccc2)n1', 'Clc1ccccc1Nc1c2oc3ccccc3c2c1C[NH+]1CCN(C(=O)c2ccccc2)CC1', 'Clc1ccccc1Nc1c2on(-c3ccccc3)c(=O)c2cc2c1CCCCC2', 'Clc1ccccc1NCc1cc[nH+]cc1-[nH+]1cc[nH+]c1', 'Clc1ccccc1NCc1cc(F)ccc1Cl', 'Clc1ccccc1NCc1ccc([S@](=O)C2CCC2)cc1', 'Clc1ccccc1NCc1cc2ccccc2n1C[C@@H]1COCCO1', 'Clc1ccccc1NCc1ccnc(OC2CCC2)c1', 'Clc1ccccc1Oc1cc(-c2ccc(O)cc2)nc2ccccc21', 'Clc1ccccc1Oc1cc(-c2n[nH]c(=O)[nH]2)ccc1O', 'Clc1ccccc1Oc1cc([N+](=O)Nc2ccccc2)ccc1N1CCN(C(=O)c2ccccc2)CC1', 'Clc1ccccc1Oc1cc([N+](=O)[O-])ccn1', 'Clc1ccccc1Oc1cc(N2Sc3ccccc3CC2)n[nH]1', 'Clc1ccccc1Oc1cc(N2CCCCC2)ncn1', 'Clc1ccccc1CSc1cncc(OCC(F)F)n1', 'Clc1ccccc1Oc1cnc(CCn2ccccc2=O)cc1', 'Clc1ccccc1Oc1cnc(C(C)C)cc1O', 'Clc1ccccc1Oc1cnc(C2CC2)n1C1CC1', 'Clc1ccccc1Oc1cnc(COc2ccccc2)[nH]1', 'Clc1ccccc1Oc1cnc(CNc2ccccc2)o1', 'Clc1ccccc1Oc1cnc(CSc2ncccn2)nc1', 'Clc1ccccc1Oc1cnc(C[NH+]2CCC(c3noc4ccccc34)CC2)cn1', 'Clc1ccccc1Oc1cnc(CBr)cc1', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CNc1nc(COc2ccccn2)cs1', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CN(CCCn1cc[nH+]c1)C[C@H]1CCCO1', 'Clc1ccccc1CN(C(=O)/C=C/c1ccc(Cl)cc1Cl)c1ccccc1', 'Clc1ccccc1CN(Cc1ccccc1)C(F)(F)F', 'Clc1ccccc1CN(C1CC1)S(=O)(=O)c1ccccc1Cl', 'Clc1ccccc1CN(C[C@@H]1CC2(COCCO2)C1)[C@@H](c1ccccc1)C1CCCC1', 'Clc1ccccc1CN(Cn1ncc2ccccc21)C1CC1', 'Clc1ccccc1CS(=O)(=O)c1ccccc1Cl', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1Cn1ncc2c1CCN(C(=O)[C@@H]1COc3ccccc3C1)C2', 'Clc1ccccc1Cn1nc2cc(-c3ccccn3)[nH]nc2c1=O', 'Clc1ccccc1CSc1ccc(CO)cc1', 'Clc1ccccc1CSc1nc2c(cnn1-c1ccccc1)c1ccccc12', 'Clc1ccccc1Oc1cc(F)c(Cn2ccc3[nH]ccc32)cc1', 'Clc1ccccc1Oc1cc(F)c(N2CCCC2)c(I)c1', 'Clc1ccccc1Oc1cc(F)c(I)cc1O', 'Clc1ccccc1Oc1cc(F)c(F)cc1F', 'Clc1ccccc1Oc1cc(F)c(-c2ncoc2C2CC2)nc1', 'Clc1ccccc1Oc1cc(F)c(Cl)cc1Cl', 'Clc1ccccc1Oc1cc(F)c(Br)cc1', 'Clc1ccccc1OC1CN(C(=O)CNc2cccc(F)c2)C1', 'Clc1ccccc1OC1CCC([NH2+]Cc2ccccc2CO)CC1', 'Clc1ccccc1OC1CCN(C(=O)c2c[nH]c3ccccc23)CC1', 'Clc1ccccc1OC1CC[NH+](CC[C@@H]2CCN(C3CC3)C2)CC1', 'Clc1ccccc1OC1CC[NH2+]CC1', 'Clc1ccccc1[C@H](c1oc2ccccc2c1[N+](=O)[O-])N1CC[NH+](Cc2ccccc2)CC1', 'Clc1ccccc1[C@H](Br)C1CCC(O)CC1', 'Clc1ccccc1[C@H](c1ccccc1)Sc1nnnn1-c1ccccc1', 'Clc1ccccc1[C@H](c1nc2ccccc2s1)N1CCc2ccccc2C1', 'Clc1ccccc1[C@H]([NH3+])[C@H]1CSC(=O)N1Cc1ccccc1', 'Clc1ccccc1[C@H]([NH3+])C[NH2+]C[C@@H]1CCCO1', 'Clc1ccccc1[C@H]([NH3+])c1cc(-c2ccc(Cl)cc2)nn1CCO', 'Clc1ccccc1[C@H]([NH3+])[C@@H]1C[NH+]2CCN1C[C@@H]2C', 'Clc1ccccc1Oc1cc(S(=O)(=O)N2CCCC2)ccc1Cl', 'Clc1ccccc1Oc1cnc2sc(C3CC3)nc2c1', 'Clc1ccccc1Oc1cnc2scc(-c3ccccc3F)c2c1', 'Clc1ccccc1Oc1cc([N+](=O)[O-])ccc1O', 'Clc1ccccc1Oc1cc([N+](=O)n2cc(Br)cn2)c[nH]1', 'Clc1ccccc1NS(=O)(=O)NC1CC1', 'Clc1ccccc1NS(=O)(=O)Nc1ccc(CC2CC2)cc1', 'Clc1ccccc1NS(=O)(=O)N1CCN(Cc2ccc(F)cc2Cl)CC1', 'Clc1ccccc1NS(=O)(=O)N[C@H]1CCS(=O)(=O)C1', 'Clc1ccccc1NS(=O)(=O)N[C@]1(CO)CCC[C@@H]1Cc1ccccc1', 'Clc1ccccc1N1Nc2c[nH+]ccc2C[C@@H]1c1cc(Cl)ccc1Cl', 'Clc1ccccc1N1Nc2c(cc(N3CCCC3)c(=O)c3ccccc32)CCC1', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1C(=O)NCC(=O)N1CCO[C@@H](c2ccc(F)cc2F)C1', 'Clc1ccccc1C(=O)N(Cc1ccncc1)Cc1ccccc1C(F)(F)F', 'Clc1ccccc1C(=O)Nc1nnc(-c2ccc(Cl)c(Cl)c2)s1', 'Clc1ccccc1C(=O)N1CCCc2ccc(F)cc21', 'Clc1ccccc1C(=O)N[C@H](C(=O)c1ccco1)[C@H]1CCCO1', 'Clc1ccccc1C(=O)N/N=C\\c1ccccc1O', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1COCCCn1cccn1', 'Clc1ccccc1COCC(=O)N1C2CC3CC(C2)CC1C3', 'Clc1ccccc1COCCc1cccc(-c2ccco2)c1', 'Clc1ccccc1COCC1CCN(C(=O)c2ccco2)CC1', 'Clc1ccccc1COCCOC[C@@H]1CCCO1', 'Clc1ccccc1COCCNS(=O)(=O)c1ccccc1', 'Clc1ccccc1COCCSc1ccc(F)cc1', 'Clc1ccccc1COCC[NH+]1C[C@@H](C)[C@H](Oc2ccccc2C(F)(F)F)C1', 'Clc1ccccc1COCCBr', 'Clc1ccccc1O/Cn1nc(-c2ccccc2F)nn1', 'Clc1ccccc1O/Cn1nnc2ccccc2c1=O', 'Clc1cccc(-c2cccc3c2O[C@@H](CN(C(C)C)C2CC3)C2)c1', 'Clc1cccc(-c2cccn2CC[NH2+]C2CC2)c1', 'Clc1ccccc1-c1cscc1-n1c([C@H]2CCCO2)cc2ccccc21', 'Clc1ccccc1-c1cscc1-n1cc(-c2cccc(Cl)c2)cn1', 'Clc1ccccc1-c1cscc1-n1cncn1', 'Clc1ccccc1-c1cscc1F\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1-c1cscc1C[NH2+]Cc1nnc2n1CCC2', 'Clc1ccccc1-c1cscc1C[NH2+]c1ccccn1', 'Clc1ccccc1-c1cscc1C[NH2+][C@H]1CCc2ccccc21', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1Cn1cnc2ccccc2c2nc(N3CCCC3)ncc21', 'Clc1ccccc1Cn1cnnc1SCc1ccccc1', 'Clc1ccccc1OCC1(C[NH2+]C2CC2)CCCCCCC1', 'Clc1ccccc1OCC1CCCC1', 'Clc1ccccc1OCC1NCCN(C(=O)Cc2ccc(C#N)cc2)CC1', 'Clc1ccccc1OCC1[NH2+]CC1', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nc(N/C=C3/CS[C@@H](F)C[NH+]3)c12', 'Clc1cccc2[nH]nc(N/N=C/Cn1)CCCC2=O', 'Clc1cccc2cc[nH]c12', 'Clc1cccc2cc(F)c(SCc3cc(C(F)(F)F)ncn3)cc12', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c12', 'Clc1cccc2cc(Br)cnc12', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc(CNC[C@@H]3COc4ccccc4O3)nn12', 'Clc1cccc2cc(C[NH+]3CCCC3)ccc12', 'Clc1cccc2cc(C[NH2+][C@H](CO)c3ccccc3)c(=O)[nH]c12', 'Clc1cccc2cc[nH]c12\n', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c12\n\n', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1ccccc1-c1nc(C(=O)CCc2cccc(-c3ccccc3)c2)n[nH]1', 'Clc1ccccc1-c1nc(Cc2ccncc2)no1', 'Clc1ccccc1-c1nc(C[S@@](=O)C/C=C/c2ccc(Br)cc2)no1', 'Clc1ccccc1-c1nc(C2CC2)no1', 'Clc1ccccc1-c1nc(CO)[nH]c2cc(-c3ccco3)[nH]c2c1N', 'Clc1ccccc1-c1nc(CN2CCN(Cc3ccnc(F)c3)CC2)c[nH]1', 'Clc1ccccc1-c1nc(C[C@@H]2COc3ccccc3O2)no1', 'Clc1ccccc1-c1nc(CCl)on1', 'Clc1ccccc1-c1nc(C[C@H]2CCCO2)n[nH]1', 'Clc1ccccc1-c1nc(C[NH+](Cc2ccco2)Cc2ccco2)co1', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c12\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cccnc1n2CCNc1ccccc1', 'Clc1cccc2cccnc12', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n', 'Clc1cccc2cc(-c3noc(C4CC4)n3)nnc12', 'Clc1cccc2cc(-n3nnnc3NCCN3CCCCC3=O)ccc12', 'Clc1cccc2cc[nH]c(=O)c12\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n', 'Clc1cccc2nn[nH]c1c1cccc2cc(Cl)c1F', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n', 'Clc1cccc2cccnc12\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n', 'Clc1cccc2c1[C@H]1C[C@@H]1C[NH+]2Cc1ccccc1C[N+]1CC1', 'Clc1cccc2cccnc12\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc1n2CC[NH2+]Cc1ccccc1', 'Clc1cccc2c(N3CCC[NH+](CC4CC4)CC3)ncnc12', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nc(Sc3nc4ccccc4s3)c12', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nc(SCc3ccsc3)c12', 'Clc1cccc2[nH]nc(SC[C@@H]3CCCO3)c12', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nc(S/c3ccc(F)c(F)c3)c12', 'Clc1cccc2[nH]nc(S(=O)(=O)N3CCCC[C@@H]3C[NH+]3CCCC3)c12', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc([C@H](C(=O)NCc3cscn3)N3CCOCC3)sc12', 'Clc1cccc2cc([C@H]3CCCCCN3C(=O)Cn3cncn3)ccc12', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc1c1nc2ccccc1', 'Clc1cccc2[nH]nnc1c1nnc(NCc3ccccc3)n12', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2c1OC[C@@H](c1ccccc1F)CN2Cc1ccccc1Cl', 'Clc1cccc2c1Oc1ccccc1CC2', 'Clc1cccc2c1O[C@@H](c1ccccc1Br)[C@@H]1C[C@@H]1[C@@H]1CCCN21', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2ncn(CCCC(=O)Nc3nncs3)c12', 'Clc1cccc2ncn(C(Cc3ccco3)c3ccccc3)c12', 'Clc1cccc2ncn(Cc3cc(F)cc(F)c3)c12', 'Clc1cccc2ncn(C3CC[NH+]=C3C(=O)NCCN2c2ccccc2Cl)c1', 'Clc1cccc2ncn(C[C@@H]3CCCCO3)c12', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2ccc(Cl)cc12', 'Clc1cccc2ccc(Br)cc1CN2C[C@@H]1C[C@@H]2CC[C@@H]1CO2', 'Clc1cccc(S[C@H]2OCc3ccccc32)c1[N+](=O)[O-]', 'Clc1cccc(S[C@H]2CCC[C@@H](OCC(F)(F)F)C2)c1', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc(/N=C/C2=CC(=O)CCC2=O)c1CN1CCC[C@@H]1CN1CCc2ccccc21', 'Clc1cccc(/N=C/c2cc([N+](=O)[O-])ccc2OC[C@@H]2CCCO2)c1', 'Clc1cccc(/N=N/CCN2CCN(c3cccc(F)c3)CC2)c1', 'Clc1cccc(/N=N/c2ccc(F)c(F)c2)c1', 'Clc1cccc(/N=N/OCC[NH+]2CCCCCC2)c1', 'Clc1cccc(/N=N/NNNc2ccccc2)c1', 'Clc1cccc(/N=N/[C@@H]2C=CCC2)c1', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CCCCC(=O)Nc1ccc(Cl)cc1F', 'Clc1ccccc1CCC(=O)Nc1ccc2c(c1)CCC2', 'Clc1ccccc1CCC1CC[NH+](CCOc2ccc(C(=O)[O-])cc2F)CC1', 'Clc1ccccc1CCCOc1cccc(Br)c1', 'Clc1ccccc1CCCNC(=O)N[C@@H](C1CCCCC1)c1ccc(F)cc1', 'Clc1ccccc1CCCn1c(C[C@@H]2CCS(=O)(=O)C2)nc2ccccc21', 'Clc1ccccc1CCCSc1nc2ccccc2[nH]1', 'Clc1ccccc1CCC[NH+]1CCC[C@@H]1c1cccs1', 'Clc1ccccc1Nc1cc([N+](=O)[O-])c(Cl)cc1Cl', 'Clc1ccccc1Nc1cc(C2CC2)ncn1', 'Clc1ccccc1Nc1cc(OCc2nc3ccccc3s2)cs1', 'Clc1ccccc1Nc1cc(I)cc(CN2CCOCC2)n1', 'Clc1ccccc1Nc1cc(NS(=O)(=O)c2ccc(F)cc2)c(Cl)cc1Cl', 'Clc1ccccc1Nc1cc(F)cc(Br)c1', 'Clc1ccccc1Nc1cc(-c2ccccc2)oc1N1CCN(c2ncccn2)CC1', 'Clc1ccccc1Nc1cc(S(=O)(=O)N2CCCC2)cc(S(=O)(=O)[O-])c1', 'Clc1ccccc1Nc1cc(Cl)ccn1', 'Clc1ccccc1Nc1cc([C@H]2CS(=O)(=O)CCN2)ncn1', 'Clc1ccccc1Nc1cc(Br)cc(F)c1F', 'Clc1ccccc1NS(=O)(=O)c1c[nH]c2cc(Cl)ccc12', 'Clc1ccccc1NS(=O)(=O)c1c[nH]nc1C(F)(F)F', 'Clc1ccccc1NS(=O)(=O)c1c[nH]c(=O)[nH]c1=O', 'Clc1ccccc1NS(=O)(=O)c1c[nH]ccc1=O', 'Clc1ccccc1NS(=O)(=O)c1c[nH]c2ccccc12', 'Clc1ccccc1NC1CCN(C(=O)c2ccc(OCc3ccccc3)s2)CC1', 'Clc1ccccc1NC1C(=O)NC(=S)N1c1ccccc1', 'Clc1ccccc1NC1C2CCC(C2)C1', 'Clc1ccccc1NC1COC1', 'Clc1ccccc1NC1CSC1', 'Clc1ccccc1NS(=O)(=O)Nc1ccc2c(c1)CCC2', 'Clc1ccccc1NS(=O)(=O)NCC[NH+]1CCCCCC1', 'Clc1ccccc1NS(=O)(=O)NCc1cccc([N+](=O)[O-])c1', 'Clc1ccccc1NS(=O)(=O)NC1CCN(O)CC1', 'Clc1ccccc1NS(=O)(=O)NC[C@@H]1CCCO1', 'Clc1ccccc1NS(=O)(=O)NC[C@H]1CCCCO1', 'Clc1ccccc1NS(=O)(=O)NC[C@]1(O)C(=O)NC([O-])C1', 'Clc1ccccc1NS(=O)(=O)NC[C@@]1(O)CCCO1', 'Clc1ccccc1Cc1noc(CCN2CCOCC2)n1', 'Clc1ccccc1CNc1n[nH]c(-c2ccccc2)n1', 'Clc1ccccc1CNc1nc2ccc(C(F)(F)F)cc2n1CC[NH+]1CCC(Cc2ccccc2)CC1', 'Clc1ccccc1CNc1nnc(Cn2ncnc2-c2cccnc2)s1', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1Cc1n[nH]c([C@H]2CCc3ccccc32)n1', 'Clc1ccccc1C(=O)OCCNC(=O)c1ccc(F)cc1', 'Clc1ccccc1C(=O)O[C@H]1CCOc2ccccc21', 'Clc1ccccc1C(=O)Oc1cccc(Cl)c1', 'Clc1ccccc1C(=O)O[C@@H]1CN(C2CC2)CCN1Cc1ccccc1', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CS(=O)(=O)C1CC[NH+](C2CCCCC2)CC1', 'Clc1ccccc1Cc1c(C(=O)[O-])ncnc1NO', 'Clc1ccccc1Cc1c(O)ccc2cccnc12', 'Clc1ccccc1Cc1c(NC(=O)N2[C@H]3CC(Cc4ccccc4)CC3=CO[C@H](c3ccc(F)cc3)N2)ccc(Cl)c1O', 'Clc1ccccc1Cc1c(F)cc(Cl)cc1F', 'Clc1ccccc1Cc1c(Cl)nc(Cl)nc1Cl', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CNc1n[nH]c(Cc2ccc(Br)cc2)n1', 'Clc1ccccc1CNc1n[nH]nc1Cc1ccc(Br)cc1', 'Clc1ccccc1C[NH+](Cc1ccccc1)CC1OCCO1', 'Clc1ccccc1Oc1cnc2nn(-c3ccccc3)nc2c1', 'Clc1ccccc1Oc1cnc2nn[n-]c2c1', 'Clc1ccccc1Oc1cnc2nnc(-c3ccc(C(F)(F)F)cc3)n2c1', 'Clc1ccccc1Oc1cnc2nnnn2c1[O-]', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1C(C)CCC[NH+]1CCCCC1', 'Clc1ccccc1C(C)(C)NCc1nc(C2CC2)n2ccccc12', 'Clc1ccccc1CNC1=N/C(=C/c2ccncc2)N1c1ccccc1', 'Clc1ccccc1CNC1CC1', 'Clc1ccccc1CNC1(c2ccccc2)CC1', 'Clc1ccccc1CNC12C[C@@H]3C[C@H](CC(C1(NC(=O)C3)C=C2)C(=O)[O-])c1ccccc1F', 'Clc1ccccc1Oc1cc(-c2c[nH]c3ccccc23)nc2cnccc12', 'Clc1ccccc1Oc1cc(-c2cccc(F)c2)ccc1O', 'Clc1ccccc1Oc1cc(Oc2cc(C(F)F)ncn2)ccc1O', 'Clc1ccccc1Oc1cc(SCC(=O)[O-])cc(Cl)n1', 'Clc1ccccc1Oc1cc(SC(F)(F)F)ccc1Cl', 'Clc1ccccc1Oc1cc(SCc2csc(C(F)(F)F)n2)[nH]n1', 'Clc1ccccc1Oc1cc(SC[C@@H]2CCOC3=c4ccccc4=[NH+][C@@H]3[C@H]2c2ccccc2)n(C2CC2)c1', 'Clc1ccccc1-c1csc(CC[NH2+]C2CCC3(CC3)C2)n1', 'Clc1ccccc1-c1csc(C(F)(F)F)n1', 'Clc1ccccc1-c1csc(Cc2ccc(F)cc2)n1', 'Clc1ccccc1-c1csc(C2CCCC2)n1', 'Clc1ccccc1-c1csc(COc2ccccc2)n1', 'Clc1ccccc1-c1csc(CN2CCN(Cc3ccccc3)CC2)n1', 'Clc1ccccc1-c1csc(C[C@@H]2C[C@@H]2c2ccccc2)n1', 'Clc1ccccc1-c1csc(Cn2ccnc2)n1', 'Clc1ccccc1-c1csc(CCl)n1', 'Clc1ccccc1-c1csc(C[NH+]2CCCCC2)n1', 'Clc1ccccc1-c1csc(C[NH2+]C2CCCCCC2)n1', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1Cn1nnc(-c2ccc(Cl)cc2)n1', 'Clc1ccccc1Cn1nnnc1SCn1cccn1', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CS(=O)(=O)N1CCC([NH2+]C2CCN(C(=O)CCCc3ccccc3Cl)CC2)C1', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1Cn1nc2cc(-c3ccccc3)ccn2c1=O', 'Clc1ccccc1C1(OCCN2CCOCC2)CC1', 'Clc1ccccc1C1(O)CC[NH+](C[C@@H]2C[C@@H]2c2ccccn2)CC1', 'Clc1ccccc1C1(Oc2ccccc2F)CC1', 'Clc1ccccc1C1(O[C@H]2CCS(=O)(=O)C2)CCCC1', 'Clc1ccccc1Oc1cnc(F)c(Cl)c1', 'Clc1ccccc1Oc1cnc(-c2n[nH]c3ccccc23)cc1', 'Clc1ccccc1Oc1cnc(-n2cnc3ccccc32)nc1', 'Clc1ccccc1Oc1cnc(Cl)cn1', 'Clc1ccccc1Oc1cnc2sn(-c3ccccc3)c2c1', 'Clc1ccccc1Oc1cnc2sncc2c1', 'Clc1ccccc1Oc1cnc2snnc2c1', 'Clc1ccccc1O/N=C(/[O-])Cc1cccc(OC2CCCC2)c1', 'Clc1ccccc1Oc1cnc(NCc2cccs2)nc1CC[NH+]1CCC[C@@H](Cl)C1', 'Clc1ccccc1Oc1cnc(Nc2ccccc2F)cn1', 'Clc1ccccc1Oc1cnc(N2CCCCC2)cc1Cl', 'Clc1ccccc1Oc1cnc(NN2CCOCC2)nc1', 'Clc1ccccc1Oc1cnc(N[C@@H]2CCC[C@@H](C(F)(F)F)C2)[nH]1', 'Clc1ccccc1Nc1ncc([N+](=O)[O-])cn1', 'Clc1ccccc1Nc1nccn1Cc1ccccc1', 'Clc1ccccc1Nc1ncccn1', 'Clc1ccccc1Nc1ncc2c(c1C(=O)[O-])C[NH+](Cc1cccnc1)C2', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1Cc1nc(-c2ccccc2Cl)no1', 'Clc1ccccc1Cc1nccc2ccc(C(=O)Nc3cnn([C@@H]4CCC[NH+](C)C4)c3)cc12', 'Clc1ccccc1Cc1nc2ccccc2nc1NC[C@H]1COCCO1', 'Clc1ccccc1Cc1ncoc1-c1cc(F)cc(F)c1', 'Clc1ccccc1Cc1ncn(-c2ccccc2)n1', 'Clc1ccccc1Cc1ncsc1Br', 'Clc1ccccc1Cn1nncc1-c1ccccc1Cn1cccn1', 'Clc1ccccc1Cn1nnc2cc(Cl)c(Cl)cc21', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1C(=O)OCC(F)(F)F', 'Clc1ccccc1C(=O)OCc1nc(Nc2ccccc2)no1', 'Clc1ccccc1C(=O)OC1(c2ccc(Cl)cc2)CC1', 'Clc1ccccc1C(=O)OC[C@@H]1CN2CCC[C@@H]2CO1', 'Clc1ccccc1C(=O)OC[C@@]1(O)CCSC1', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1C[NH+](CCCN1CCOCC1)C1CC1', 'Clc1ccccc1C[NH+](Cc1ccccc1Cl)[C@@H]1CCN(C(=O)OC(F)(F)F)C1', 'Clc1ccccc1C[NH+](C1CC1)[C@H]1CCCc2ccccc21', 'Clc1ccccc1C[NH+](C[C@@H]1CCCO1)[C@H]1CC[C@](O)(COCC(F)F)C1', 'Clc1ccccc1C[NH+](C[C@H]1C[C@@H]2CCN1CCO2)C[C@@H]1CCCO1', 'Clc1ccccc1C[NH+](C[C@]1(Cc2ccccc2)O[C@H]1CCO)C1CC1', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc(OC[C@@H]3CCCO3)oc12', 'Clc1cccc2cc(O)ccc12', 'Clc1cccc2cc(Oc3nc(-c4ncn[nH]4)cs3)ncc12', 'Clc1cccc2cc(O[C@H](CSC(F)(F)F)C(F)F)ncc12', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2c(N[C@@H]([C@@H]3CCCO3)[C@H]3CCS(=O)(=O)C3)cccc12', 'Clc1cccc2ccc(NCCC[NH+]3CCCC3)nc12', 'Clc1cccc2ccc(Nc3ccccc3C(=O)OC[C@H]3CCCO3)c2c1', 'Clc1cccc2ccc(N3CCO[C@@H](c4cc(Cl)cc(Cl)c4)C3)nc12', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc(Br)sc12', 'Clc1cccc2cc(Br)[nH]c(=O)c12', 'Clc1cccc2cc(Br)ccc12', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc(F)[nH]c12', 'Clc1cccc2cc(F)ncc12', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc1n2-c1ccccc1', 'Clc1cccc2cccnc1n2CCC(=O)N[C@H](c1ccccc1)C1CCCC1', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc(F)ncc12', 'Clc1cccc2cc(F)nnc12', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc(Br)c(/C=C/C(=O)[O-])cc12', 'Clc1cccc2cc(Br)ccc12', 'Clc1cccc2cc(Br)cnc12', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2ccc(/C=C3\\SC(=S)N(c4nonn4)C3)cc12', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc(F)[nH]/c(=N/CCCNC(=O)c3ccc(OC(F)F)cc3)c12', 'Clc1cccc2cc(F)[nH]c12', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc(/C=C/Br)ccc12', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2c(N)[nH]nc12', 'Clc1cccc2c(N)cccc12', 'Clc1cccc2ncncc12', 'Clc1cccc2ncnc(Sc3ccccc3)c12', 'Clc1cccc2ncnc(N3CCO[C@H]4CCCC[C@@H]43)c12', 'Clc1cccc2ncncc1n2CCOc1ccccc1', 'Clc1cccc2ncncc12', 'Clc1cccc2ccc(S[C@H](Cc3ccco3)[NH+]3CCCC3)cc12', 'Clc1cccc2ccc(SCCC[NH+]3CCC[C@@H](n4cccn4)C3)nc12', 'Clc1cccc2nn(S[C@@H](c3ccccc3)c3ccc4ccsc4c3)nc12', 'Clc1cccc2[nH]nc(N)c12', 'Clc1cccc2[nH]nc(NCCC3=CCCOC3)n2n1', 'Clc1cccc2[nH]nc(NC(=O)Cc3cccnc3)c12', 'Clc1cccc2[nH]nc(NCc3ccc(Oc4ccccc4)cc3)c12', 'Clc1cccc2[nH]nc(NC[C@@H](c3ccsc3)c3ccccc3)c12', 'Clc1cccc2[nH]nc(NC[C@H](O)c3ccccc3)c12', 'Clc1cccc2nn(S[C@@H](Cc3cccnc3)c3ccccc3)nc12', 'Clc1cccc2nn(S[C@@H](NC3=CCCC3)C(F)(F)F)nc12', 'Clc1cccc2cc([C@H](Cl)c3ccc(OCc4ccccc4F)cc3)sc12', 'Clc1cccc2cc([C@H](Br)[C@@H]3COc4ccccc4O3)ccc12', 'Clc1cccc2cc([C@H](O)CSc3nc4ccccc4s3)oc12', 'Clc1cccc2c1C/C(=C\\c1ccccc1O)[C@@H]2c1ccccc1F', 'Clc1cccc2c1CCC[C@H]2[NH2+][C@@H](c1cccc(CC(F)F)c1)[C@H]1CCS(=O)(=O)C1', 'Clc1cccc2c1CNC(=O)C[C@@H]2c1ccccc1C(F)(F)F', 'Clc1cccc2c1C[C@@H](Cc1cccnc1)C2', 'Clc1cccc([S@@](=O)Cc2cccc(I)c2)c1', 'Clc1cccc([S@@](=O)/C=C/c2ccc([N+](=O)[O-])cc2)c1', 'Clc1cccc([S@@](=O)c2nnc(N3CCCC3)o2)c1', 'Clc1cccc([S@@](=O)[C@@H]2CC[C@H](Nc3ccccc3F)C2)c1', 'Clc1ccccc1NC(=S)NCc1cccc(C(F)(F)F)c1', 'Clc1ccccc1NC(=S)Nc1ccc(F)cc1', 'Clc1ccccc1NC(=S)N1/C(=N/N=C\\c2ccc(C(F)(F)F)s2)SC1=S', 'Clc1ccccc1NC(=S)NNc1ccc(F)cc1F', 'Clc1ccccc1O/N=C1\\SC(=S)N(CC[NH2+]C2CCCCC2)[C@H]1c1cccc(Br)c1', 'Clc1ccccc1[C@H]1CCC[NH+]1CCCN1CCN(c2cccc[nH+]2)CC1', 'Clc1ccccc1[C@H]1C(c2ccccc2)=Nc2ncnn21', 'Clc1ccccc1[C@H]1CN(C(=O)Cn2nnnc2-c2ccccc2)CCO1', 'Clc1ccccc1[C@H]1C[C@@H](c2cccc(F)c2)CS1', 'Clc1ccccc1[C@H]1C[C@H](CCCO)CN1C(=O)c1cc2c([nH]c1=O)CCC2', 'Clc1cccc(Br)c1O', 'Clc1cccc(Br)c1OCc1ccco1', 'Clc1cccc(Br)c1Oc1cccc2c1CCC2', 'Clc1cccc(Br)c1ONCc1ccc[nH]1', 'Clc1cccc(Br)c1O[C@@H]1c2ccccc2C[C@H]1O', 'Clc1cccc(Br)c1O[P@](=O)(=O)c1ccccc1', 'Clc1cccc(Br)c1O[C@H](C(=O)c1cccnc1)c1cccs1', 'Clc1cccc(Br)c1O/C=C/C(=O)c1ccc(Cl)cc1Cl', 'Clc1cccc(F)c1-c1cc(Cl)cc(Cl)c1', 'Clc1cccc(F)c1Br\n\n', 'Clc1cccc(S(=O)(=O)N(CCSc2nnc(C3CCCC3)n2C2CC2)C2CC2)c1', 'Clc1cccc(F)c1[C@@H](CCn1cccn1)N[C@@H](c1ccccc1)c1ccccc1', 'Clc1cccc(F)c1[C@@H](c1ccccc1Br)N1CCOCC1', 'Clc1cccc(F)c1[C@@H]([O-])[C@H](CSc1ccc(F)cc1)N1CCCC1=O', 'Clc1cccc(S(=O)(=O)N2CCC(C(=O)OCc3ccsc3)CC2)c1', 'Clc1cccc(S(=O)(=O)Nc2ccc(C[NH+]3CCCC3)cc2)c1', 'Clc1cccc(S(=O)(=[NH2+])CC2C(C)(C)CS2)c1', 'Clc1cccc(Br)c1Cl\n', 'Clc1cccc(Br)c1Br\n', 'Clc1cccc(F)c1-n1cnnc1', 'Clc1cccc(Br)c1Cl\n\n', 'Clc1cccc(Br)c1Br\n\n', 'Clc1cccc(Nc2cc([N+](=O)[O-])ccc2Cl)c1', 'Clc1cccc(Nc2cc(N[C@H]3CCOC3)nc(-c3ccccc3)n2)c1', 'Clc1cccc(Nc2cc(F)ccc2Oc2cccc(F)c2)c1', 'Clc1cccc(Nc2cc(-c3cccs3)no2)c1', 'Clc1cccc(Nc2cc(S(=O)(=O)N3CCCCC3)ccc2Cl)c1', 'Clc1cccc(Nc2cc(Cl)ccc2O)c1', 'Clc1cccc(Nc2cc(Br)ccc2F)c1', 'Clc1cccc(Nc2cc(Br)ccc2OC[C@H]2CCCO2)c1', 'Clc1cccc([C@@H]2[C@@H](c3ccc(Cc4cnn[nH]4)nc3)N=C2Cl)c1', 'Clc1cccc([C@@H]2[C@@H]3CCC[C@@]3(CO)C2)c1', 'Clc1cccc(Br)c1Cl\n\n\n', 'Clc1cccc(F)c1Br\n\n\n', 'Clc1cccc(Br)c1Br\n\n\n', 'Clc1cccc(F)c1Br\n\n\n\n', 'Clc1cccc(Nc2cc(Cl)ccc2Cl)c1', 'Clc1cccc(Nc2cc(F)cc(Br)c2)c1', 'Clc1cccc(Nc2cc([N+](=O)[O-])ccc2Br)c1', 'Clc1cccc(NCCC[NH+](CCn2ccn[nH+]2)C2CC2)c1', 'Clc1cccc(NCC(=O)Nc2ccncc2C(F)(F)F)c1', 'Clc1cccc(NCCc2nccc(-c3cccc([N+](=O)[O-])c3)n2)c1', 'Clc1cccc(NCC2(O)CCOCC2)c1', 'Clc1cccc(NCCn2cc([N+](=O)[O-])cn2)c1', 'Clc1cccc(NCCS(=O)(=O)/N=C/c2ccccc2)c1', 'Clc1cccc(NCC[NH+]2CCC[C@@H]2CN2CCOCC2)c1', 'Clc1cccc(NCC[NH2+]Cc2sccc2Cl)c1', 'Clc1cccc(-c2n[n-]c(Sc3cnnnc3)n2)c1', 'Clc1cccc(-c2n[n-]nc2CN2CCCC[C@H]2c2ccccc2)c1', 'Clc1cccc(Br)c1Cl\n\n\n\n', 'Clc1cccc(F)c1-c1[nH+]ccn1CCC[NH+]1CCCC1', 'Clc1cccc(F)c1-c1ccc(Br)c(C(=O)[O-])c1', 'Clc1cccc2cc(CNC[C@@H]3CCc4ccccc4O3)c(F)cc12', 'Clc1cccc2cc(CN(Cc3ccnc4ccccc34)Nc3ccc(Br)cc3)sc12', 'Clc1cccc2cc(CNc3nccs3)ccc12', 'Clc1cccc2cc(CNS(=O)(=O)c3ccccc3Cl)[nH]c12', 'Clc1ccccc1Nc1ncnc(N2CCC(c3ccc(F)cc3)CC2)n1', 'Clc1ccccc1Nc1nc2ccccc2cc1CO', 'Clc1ccccc1Nc1nnnn1-c1cccc2ccccc12', 'Clc1ccccc1Nc1nn[n-]c1COc1ccccc1', 'Clc1ccccc1Nc1nn[n-]n1', 'Clc1ccccc1OC1(OC2CCCC2)CCOCC1', 'Clc1ccccc1OC1(O)CCN(c2cccc[nH+]2)CC1', 'Clc1ccccc1OC1(Oc2cnc(-c3cscn3)s2)CCOCC1', 'Clc1ccccc1NCC(=O)N1CCN(S(=O)(=O)c2ccc(F)cc2)CC1', 'Clc1ccccc1NCC(C1CC1)S(=O)(=O)c1ccc(Cl)cc1', 'Clc1cccc(Br)c1Br\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n\n\n', 'Clc1cccc(Br)c1Br\n\n\n\n\n', 'Clc1cccc([S@@]c2nnnn2-c2ccccc2)c1', 'Clc1cccc(Br)c1Cl\n\n\n\n\n\n', 'Clc1cccc([S@@]c2nc3cc(Cl)ccc3[nH]c2=O)c1', 'Clc1cccc([S@@]c2ncc(Br)cc2Cl)c1', 'Clc1cccc([S@@]c2ncnc(Nc3ccc(Br)cc3Cl)c2[O-])c1', 'Clc1cccc(Br)c1N[C@@H](c1ccc(F)cc1)N1CCCC1', 'Clc1cccc(Br)c1N[C@@H]1CCc2ccccc21', 'Clc1cccc(Br)c1Br\n\n\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n\n\n\n\n', 'Clc1cccc([S@@]CCCCC(=O)Nc2ccc3c(c2)OCCC3)c1', 'Clc1cccc([S@@]CC(F)F)c1', 'Clc1cccc([S@@]CCc2ccc(CO)cc2)n1', 'Clc1cccc([S@@]CC2(Cn3cc(Cl)cn3)CC2)c1', 'Clc1cccc([S@@]CC=N/c2ccc(Cl)cc2Cl)c1', 'Clc1cccc([S@@]CCOc2ccc(Br)cn2)c1', 'Clc1cccc([S@@]CC[C@@H]2CCC[NH2+]C2)c1', 'Clc1cccc([S@@]CCn2ccnc2OCC(F)F)c1', 'Clc1cccc([S@@]CCS(=O)(=O)c2ccc(F)cc2)c1', 'Clc1cccc(Br)c1Br\n\n\n\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n\n\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n\n\n\n\n\n\n', 'Clc1cccc(Br)c1Br\n\n\n\n\n\n\n\n', 'Clc1cccc(C(=O)NC2CC2)c1', 'Clc1cccc(C(F)(F)F)c1', 'Clc1cccc(C(Br)CCl)c1', 'Clc1cccc(CCC[NH+]2CCCCCC2)c1', 'Clc1cccc(CC(=O)N[C@H]2CCCC[C@@H]2CO)c1F', 'Clc1cccc(CCc2nnc(Cc3ccsc3)n2Cc2ccccc2)c1', 'Clc1cccc(CC2(CO)OCCO2)c1', 'Clc1cccc(CCOC(F)(F)F)c1', 'Clc1cccc(CCNC(=O)N[C@@H]2CCC[C@H](C3CC3)C2)c1', 'Clc1cccc(CCn2ccc(C3CCC3)n2)c1', 'Clc1cccc(CC[C@H](CO)Nc2ccccc2)c1', 'Clc1cccc(CC[NH+]2CCN([C@@H]3CCN(CC4(O)CCOCC4)CCCC3)CC2)c1', 'Clc1cccc(CC/C([O-])=C2/NC=C/c3ccccc32)c1', 'Clc1cccc(CC[NH2+]C2CCN(CCO)CC2)c1', 'Clc1cccc(Br)c1Cl\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc(Br)c1NCCCOc1ccccc1F', 'Clc1cccc(Br)c1NCc1ccccc1Oc1ccccc1', 'Clc1cccc(Br)c1NC1=NS(=O)(=O)CC=CC1', 'Clc1cccc(Br)c1NC[C@@H]1CCCc2ccccc21', 'Clc1cccc(Br)c1NC[C@H]1CC[C@H](c2ccccc2)[NH2+]1', 'Clc1cccc(Br)c1NC[C@]1(O)CCSC1', 'Clc1cccc(S[C@H](C[C@@H]2CCCCC[NH2+]2)c2cccs2)c1', 'Clc1cccc2cc(O)[nH]c12', 'Clc1cccc2cc(O)c(F)cc12', 'Clc1cccc2cc(Br)sc12', 'Clc1cccc(/C=C/C[NH2+][C@@H](C(=O)[O-])C2CCCC2)c1', 'Clc1cccc(/C=C2\\SC(=S)NC2=O)c1', 'Clc1ccccc1O/C=C[C@@H](O)COC1CCCC1', 'Clc1ccccc1O/C=C[C@@H]1CN2CCCC[C@H]2CO1', 'Clc1ccccc1-c1n[nH]c2c1CCCC2', 'Clc1ccccc1-c1n[nH]c2ccccc12', 'Clc1ccccc1-c1n[nH]c2cnn(Cc3cccs3)c12', 'Clc1ccccc1-c1n[nH]c2cc[nH]c12', 'Clc1ccccc1-c1n[nH]c2ccccc12', 'Clc1ccccc1-c1n[nH]cc1-c1ccc(Cl)cc1Cl', 'Clc1ccccc1-c1n[nH]cc1C(=O)CN1C(=O)COc2ccc(F)cc21', 'Clc1ccccc1-c1n[nH]c2cn[nH]c(=O)c12', 'Clc1ccccc1-c1n[nH]c2cncc(NCC[NH+]3CCCCC3)c12', 'Clc1ccccc1-c1nn(CCCc2ccccc2)c2c1CCC2', 'Clc1ccccc1-c1nn(Cc2ccccc2)nc1N1CCC(c2ccccc2)CC1', 'Clc1ccccc1-c1nn(C2CC2)c(-n2nc([O-])c3ccccc23)n1', 'Clc1ccccc1-c1nn(CN2CCc3ccccc32)c2c1CCCC2', 'Clc1ccccc1-c1nn(C[NH+]2CCN(C[C@@H](O)c3ccccc3)[C@@H](O)C1)C2', 'Clc1ccccc1OCc1ccc(Cl)cc1', 'Clc1ccccc1OCc1cccc(Oc2ccc(-n3cccn3)cc2)c1', 'Clc1ccccc1OCc1ccc2c(c1)CCC(=O)N2', 'Clc1ccccc1OCc1ccc(C(=O)NCc2ccco2)cc1', 'Clc1ccccc1OCc1ccc(Oc2ncnc(Nc3cccc(F)c3)c2)cc1', 'Clc1ccccc1OCc1ccc(N2CCOCC2)cc1', 'Clc1ccccc1OCc1ccc(F)cc1NC(=O)c1ncn(Cc2ccccc2)n1', 'Clc1ccccc1OCc1ccc(-c2cccc3ccccc23)o1', 'Clc1ccccc1OCc1ccc(Cl)cc1Cl', 'Clc1ccccc1OCc1ccc(Br)c(N2CCOCC2)c1', 'Clc1ccccc1OC1(NC(=O)c2ccc(Cl)cc2F)CC1', 'Clc1ccccc1OC1(Nc2c(F)ccc(Br)c2)c2ccccc2O1', 'Clc1ccccc1OC1(N2CCOC[C@@H]2c2ccccc2)CC1', 'Clc1ccccc1OCc1ccn(CCCOC(F)F)n1', 'Clc1ccccc1OCc1ccn([C@@H]2CCSC2)c1', 'Clc1ccccc1OCc1ccn(-c2cccc(F)c2Cl)n1', 'Clc1ccccc1OCc1ccn(Cl)n1', 'Clc1ccccc1OCc1ccn([C@H]2CCS(=O)(=O)C2)[nH+]1', 'Clc1ccccc1Oc1cc(Cl)cc(Cl)c1', 'Clc1ccccc1Oc1cc(Cl)ccc1N1CCCCC1', 'Clc1ccccc1Oc1cc(Cl)ccn1', 'Clc1ccccc1Oc1cc(Br)cc(F)c1', 'Clc1ccccc1Oc1cc(Br)ccc1O', 'Clc1ccccc1Oc1cc(Br)cc2ccccc12', 'Clc1ccccc1Oc1cc(OCC(F)(F)F)ccc1O', 'Clc1ccccc1Oc1cc(OC(c2ccccc2)c2ccccc2)cs1', 'Clc1ccccc1Oc1cc(OCc2ccc(Cl)cc2)cc2c1OCO2', 'Clc1ccccc1Oc1cc(OC2CCCCC2)c[nH]c1=O', 'Clc1ccccc1Oc1cc(OC[C@H]2CCCO2)nc(-c2ccco2)n1', 'Clc1ccccc1Oc1cc(-c2nc3c([nH]c2=O)CCCC3)ccc1O', 'Clc1ccccc1Oc1cc(-c2noc(-c3cccnc3)n2)ncn1', 'Clc1ccccc1Oc1cc(-c2nnc(-c3cccs3)o2)ccc1F', 'Clc1ccccc1Oc1cc(-c2n[nH]c(COc3ccccc3)n2)c[nH]1', 'Clc1ccccc1Oc1cc(NCCc2nc3ccccc3o2)ccn1', 'Clc1ccccc1Oc1cc(NC(=S)Nc2ccccc2)ccc1Cl', 'Clc1ccccc1Oc1cc(NCc2cccnc2)ncn1', 'Clc1ccccc1Oc1cc(NC2(C(=O)[O-])CCOCC2)ccc1F', 'Clc1ccccc1Oc1cc(NC[C@H]2CCCO2)ncn1', 'Clc1ccccc1Oc1cnccn1', 'Clc1ccccc1Oc1cnccc1C(=O)N1CCCC[C@@H]1C#CCCl', 'Clc1ccccc1Oc1cnccn1', 'Clc1ccccc1Oc1cnc2cc([N+](=O)[O-])cnc2n1', 'Clc1ccccc1Oc1cnc2cc(C(=O)NCc3ccccc3)[nH]c2c1', 'Clc1ccccc1Oc1cnc2cc(F)c(F)cc2c1[O-]', 'Clc1ccccc1Oc1cnc2cc(-c3cccc(C4O[C@H](CO)C4)c3)[nH]c2n1', 'Clc1ccccc1Oc1cnc2cc(S(=O)(=O)N3CCN(c4cccc(Br)c4)CC3)ncc2n1', 'Clc1ccccc1Oc1cnc2cc([C@H]3CCCN(C(=O)c4ccccc4)C3)[nH]c2c1', 'Clc1ccccc1Oc1cnc2cc(Br)ccc2n1', 'Clc1ccccc1OCC[NH+]1CCC[C@H]1c1ccc(F)cc1', 'Clc1ccccc1O/N=C(/c1ccncc1)S(=O)(=O)c1cccc(Cl)c1Cl', 'Clc1ccccc1O/N=C(/OCc1cccc([N+](=O)[O-])c1)n1cncn1', 'Clc1ccccc1O/N=C(/NCc1cncc(C(F)(F)F)c1)N1CCOCC1', 'Clc1ccccc1O/N=C(/Cl)c1nccs1', 'Clc1ccccc1O/N=C(/[O-])C[C@H]1Cc2ccccc2S1', 'Clc1ccccc1[C@H]1[C@@H](C2CC2)Nc2ncnn21', 'Clc1ccccc1[C@H]1[C@@H](NC(=O)c2ccc3c(c2)CCCC3)CC[NH+]1Cc1ccccc1', 'Clc1ccccc1[C@H]1[C@@H](n2cccn2)CCC[NH+](CCc2ccccc2)C1', 'Clc1ccccc1[C@H]1[C@@H]([O-])CC(O)=N[C@@H]1N1CCOCC1', 'Clc1ccccc1[C@H]1[C@@H]([C@H]2CCS(=O)(=O)C2)NC2=C1C1CC[NH+](CCc3ccc(F)cc3)C=C12', 'Clc1ccccc1[C@H]1[C@@H]([NH2+]Cc2ccc3c(c2)OCO3)CCCC1(CCO)C[C@@H]1OCCO1', 'Clc1ccccc1Nc1cnc(CCN2CCOCC2)[nH]1', 'Clc1ccccc1Nc1cnc([S-])nc1', 'Clc1ccccc1Nc1cnc(Oc2ccc(Cl)cc2)nc1', 'Clc1ccccc1Nc1cnc(NCc2cccnc2)nc1', 'Clc1ccccc1Nc1cnc(I)c(Br)c1', 'Clc1ccccc1Nc1cnc(F)cc1Cl', 'Clc1ccccc1Nc1cnc(-c2ccccc2)o1', 'Clc1ccccc1Nc1cnc(S(=O)(=O)N(C(C)C)N2CCOCC2)c2ccccc12', 'Clc1ccccc1Nc1cnc(Cl)cc1I', 'Clc1ccccc1Nc1cnc([O-])nc1C1CC1', 'Clc1ccccc1Nc1cnc(Br)s1', 'Clc1ccccc1N1N(Cc2ccccc2)C[C@@H]2CCCC[C@@H]21', 'Clc1ccccc1N1N(CCCc2nnc3cccsc23)CCC1', 'Clc1ccccc1N1N(CC(=O)NCc2ccc(F)cc2)Nc2ccccc21', 'Clc1ccccc1N1N(CCc2cccc(Cl)c2)C=C1COc1ccccc1F', 'Clc1ccccc1N1N(CC2CC2)C(=O)N1Cc1cccc(F)c1', 'Clc1ccccc1N1N(CCOc2ccc(Br)cc2Cl)CCCC1', 'Clc1ccccc1N1N(CCNC(=O)c2cccc3ccccc23)CC1', 'Clc1ccccc1N1N(CCn2cc[nH+]c2-c2ccco2)CC1', 'Clc1ccccc1N1N(CCSc2ccc([N+](=O)[O-])cc2)CC1', 'Clc1ccccc1N1N=C(Sc2ccccc2Cl)SC1=S', 'Clc1ccccc1N1N=C(C(F)F)[C@@H](c2cccc(F)c2)C(=O)N1c1ccccc1Cl', 'Clc1ccccc1N1N=C(N2CCN(c3ccccc3)CC2)SC1=S', 'Clc1ccccc1N1N=C(c2cccnc2Cl)C[NH+]1Cc1ccc(F)cc1', 'Clc1ccccc1OCCn1cccc(Br)c1=O', 'Clc1ccccc1OCCn1ncnc1-c1cc(Cl)ccc1O', 'Clc1ccccc1OCC[NH2+]Cc1cn(-c2ccccc2)nn1', 'Clc1ccccc1OCC[NH2+][C@@H]1CCc2ccccc21', 'Clc1ccccc1Oc1cc([N+](=O)[O-])ccc1Cl', 'Clc1ccccc1Oc1cc(Cl)c([N+](=O)[O-])cc1[N+](=O)[O-]', 'Clc1ccccc1Oc1cc(Cl)c(CO)cc1', 'Clc1ccccc1Oc1cc(Cl)c(N2CCOCC2)c(F)c1', 'Clc1ccccc1Oc1cc(Cl)c(Cl)cc1Cl', 'Clc1ccccc1Oc1cc(Cl)c(Br)s1', 'Clc1ccccc1Oc1cc(Br)c([N+](=O)[O-])cc1Br', 'Clc1ccccc1Oc1cc(Br)c(COc2ccc(Br)cc2)nc1N1CCOCC1', 'Clc1ccccc1Oc1cc(Br)c(N2CC[NH+](CC(=O)NC(C)(C)C)CC2)cc1', 'Clc1ccccc1Oc1cc(Br)c(I)c(F)c1', 'Clc1ccccc1Oc1cc(Br)c(F)cc1Cl', 'Clc1ccccc1Oc1cc(Br)c(Cl)cc1Cl', 'Clc1ccccc1Oc1cc(Br)c([O-])c(Br)c1']
num_valid: 10003
num_all_valid: 11886
sorted_smiles= ['O=C(/C(c1cccs1)N1CCN(c2ccccc2OCc2c[nH]c3ccccc23)CC1)c1ccc2ccccc2c1', 'Clc1cccc2c1NC(=O)N2[C@H](C(=O)N(Cc1cccc(F)c1)c1ccccc1)c1ccccc1', 'Clc1ccc([C@@H]([O-])CNc2ccc3ccc4ccn4c(=O)n(Cc4cccc(F)c4)n3c2)cc1', 'Oc1ccc(Oc2cc([C@@H]3OCCN(c4ccccc4OC(F)(F)F)n3Cc3ccccc3)ccc2F)cc1', 'Oc1ccccc1-c1c(C(=O)NCc2ccccn2)cc(-c2ccccc2)n1Cc1ccccc1', 'O=C(/C(c1ccccc1Cl)N1CCN(c2ccccc2)CC1)c1cccc2c1ccc1ccccc21', 'O=C(/N=C/c1ccc2c(c1)N(CCc1ccccc1)c1ccccc1-2)c1ccc[nH]1', 'Oc1ccc(/C=C(\\Cn2c3ccccc3c3ccccc32)c2ccccc2Cl)cc1', 'O=C(c1cn(-c2ccccc2)nc1-c1cc2ccccc2nc1N1CCOCC1)c1ccc2ccccc2c1', 'Oc1ccc([C@H](/N=C2\\Oc3ccccc3C2=O)c2ccccc2Cl)cc1OCc1ccccc1', 'O=C(c1cn2ccsc2c1)N1CCC(c2[nH+]ccn2CCC(c2ccccc2)c2ccccc2)CC1', 'Clc1nnnc(CN2C(=O)/C(=C/c3cccc4ccccc34)c3ccccc32)c1-c1ccccc1', 'O=C(c1cc(Cl)ccc1N1CC[NH+](C2CC2)CC1)N(Cc1ccsc1)C[C@@H]1CCCO1', 'Clc1ccc([C@@H](N[C@@H](c2ccccc2)c2ccccc2)c2ccccc2)cc1', 'O=C(CC(c1ccccc1)N(C1CC1)c1ccccc1)N1CC[C@@]2(CCC(=O)Nc3ccccc32)C1', 'O=C(CNc1nn(-c2ccccc2)c2ccccc12)Nc1cnn(Cc2ccccc2)c1', 'Oc1cc(O[C@@H]2Oc3ccccc3Oc3ccccc32)ccc1N1CCC[NH+]1CCc1ccccc1', 'Clc1ccccc1C1=C(c2ccc(OCn3cccn3)cc2)Nc2ccccc21', 'O=C([C@H]1c2ccccc2C(=O)NC12CCCN(c1cccc(Cl)c1)C2=O)Nc1cc(Cl)ccc1Cl', 'ClC(c1cc2ccccc2c2onc(-c3ccccn3)c12)[C@H]1NC(=O)N(Cc2ccccc2F)C1', 'Clc1ccccc1CNC12C[C@@H]3C[C@H](CC(C1(NC(=O)C3)C=C2)C(=O)[O-])c1ccccc1F', 'Clc1ccccc1Cc1c(NC(=O)N2[C@H]3CC(Cc4ccccc4)CC3=CO[C@H](c3ccc(F)cc3)N2)ccc(Cl)c1O', 'O=C(Cc1c(N[O-])c2ccccc2c(=O)n1C1CCCC1)N1CCN(Cc2ccc3c(c2)OCO3)CC1', 'O=C(c1cn2ccsc2c1Nc1ccc(Br)cc1F)N1CCO[C@@H](c2ccccc2)C1', 'O=[N+]([O-])c1c(-c2nnnn2-c2ccccc2)nc(-c2ccccc2)c2ccccc12', 'Clc1ccc(-c2ccccc2N2[C@H](Cl)[C@H](c3ccccc3)C3=C2CCCC3)cc1', 'O=C([O-])c1nn2c(-c3cccnc3)cc(-c3ccc(Cl)cc3)nc2c1-c1ccccc1F', 'O=C([C@H](CN1CCOCC1)N1CCN(c2cccc(Cl)c2)CC1)C[NH+](CC1CC1)C1CC1', 'Oc1cccc(-n2ccc(-c3cnc(-c4cc5ccccc5[nH]4)cc3)n2)c1', 'O=C(/C(CCCN1C(=O)NC2(CCCCCC2)N1c1ccccc1)Oc1ccccc1)c1ccccc1Cl', 'O=[N+]([O-])c1c(-c2nc3ccccc3o2)ccc2nccc(NCc3ccccc3)c12', 'O=C(/C(c1cccs1)(c1ccccc1)c1ccc2c(c1)C=CC2)c1ccccc1', 'Oc1cccc(N2CCN(C(=O)c3ccc(C[NH+]4CCC(C(=O)Nc5cccc(Cl)c5)cc4)cc3Cl)CC2)c1', 'Clc1ccccc1Nc1c2oc3ccccc3c2c1C[NH+]1CCN(C(=O)c2ccccc2)CC1', 'CNC(=O)N[C@H]1CC(=O)N2c3ccccc3C(=O)N2[C@H](c2ccccc2)[C@@H]1C(=O)c1cccc(Br)c1', 'Clc1ccccc1C[NH+](C[C@H]1C[C@@H]2CCN1CCO2)C[C@@H]1CCCO1', 'Clc1ccccc1[C@H]1N/C(=C/c2ccc(-c3ccco3)cc2)N1c1ccc(F)cc1', 'Oc1cc(-n2nnnc2-c2c(F)cccc2F)c2ccccc2c1OCc1c(F)ccc(Br)c1', 'Clc1ccc(Br)cc1NCc1ccc2c(c1)CCC(=O)N2Cc1cc(F)ccc1F', 'Oc1cc(COCc2cccc(Br)c2)c2c(cc3ccccc32)c2ccccc12', 'O=C(/C(c1cccs1)[NH+]1CCN(C(=O)c2cnn(-c3ccccc3)c2-c2ccccc2)CC1)c1ccc(C(F)(F)F)cc1', 'O=C(/N=c1\\[nH]c2ncnc(-c3ccccc3)c2c2ccc(O)cc12)c1cncs1', 'Oc1ccccc1/N=N\\C(=N\\OCc1ccccc1)c1nc(-c2ccc(-c3ccccc3)cc2)no1', 'Clc1ccccc1[C@H]1[C@@H]([NH2+]Cc2ccc3c(c2)OCO3)CCCC1(CCO)C[C@@H]1OCCO1', 'O=C(/C=C1/c(=C/C=C/C2=CCCCC2)[C@H]1c1cccc(Cl)c1)c1ccc2c(c1)OCO2', 'O=[N+]([O-])c1ccc2cc[nH]c2c1CN1CCN(c2ccccc2)[C@@H]1c1ccc(-c2ccc(F)cc2)cn1', 'Clc1ccccc1Oc1cc(-c2c[nH]c3ccccc23)nc2cnccc12', 'O=C(/C=C/c1c(-c2ccccc2)[n+](-c2ccccc2)c([O-])[nH]c1=O)Nc1cccc(Cl)c1', 'O=C(CC[C@@H]1c2ccccc2C(=O)N1Cc1cccc(C(F)(F)F)c1)N1CCN(c2cccc[nH+]2)CC1', 'O=C(/N=C/CC1=Cc2ccccc2Oc2ccccc21)c1ccc(-c2cccs2)cc1', 'Clc1cccc2cc(CN(Cc3ccnc4ccccc34)Nc3ccc(Br)cc3)sc12', 'O=C(/N=c1/[nH]ccn1C1CC1)Nc1ccccc1N1CCN(c2ccccc2F)CC1', 'O=C(c1cn2ccsc2c1)N1CCn2nc(-c3ccc(F)cc3)cc2[C@@H]1c1ccccc1', 'O=C(c1c[nH]nc1-c1cccc2c1cnn2-c1ccccc1)Nc1cc(F)ccc1F', 'ClC[C@@H]1OCCN(C(=O)Cc2ccccc2)C1', 'O=C(C1CC[NH+]([C@H]2CC[NH+](Cc3c[nH]c4ccccc34)CC2)C1)NCCOc1ccccc1', 'O=C([C@H](c1ccc2c(c1)OCCCO2)N1CCOCC1)N1CCN(Cc2ccccc2)CC1', 'Oc1ccc(Oc2ccc(C(=O)Nc3ccccc3)c(-c3ccccc3)n2)cc1', 'Oc1ccc(/N=c2[nH]c3ccccc3n2Cc2ccccc2)cc1OCC1CCN(CO)CC1', 'Oc1ccc(Oc2cnc(Oc3ccccc3)nc2-c2ccccc2)cc1', 'O=C(/C(c1cccs1)[C@@H](c1ccco1)N1C2=C(CCCC2=O)c2ccccc21)N1CCNC1=O', 'O=C(COc1cc(Cl)c2[nH]c3c(c2c1)CCCC3)NCC1CC[NH+](Cc2ccccc2)CC1', 'Clc1ccc(-c2nn(-c3ccc(Br)cc3)c3ccccc23)cc1', 'O=C(c1c[nH]c2ccccc12)N1CCC[C@H]1CN1CCc2ccccc21', 'O=C([C@H]1c2ccccc2C[C@@H]1Oc1ccc2c(c1)OCCO2)N1CCN(Cc2cn3ccccc3[nH+]2)CC1', 'Clc1cccc([C@@H]2CC[C@@H]2CN2CCO[C@@H](c3ccccc3)C2)c1', 'Oc1ccc([C@H](/N=C2\\Oc3ccccc3O2)c2c[nH]nc2-c2cc(F)ccc2F)cc1', 'Clc1ccccc1[C@H]1[C@@H]2O[C@@H](COC(=O)c3ccccc3)[C@H]2C(=O)N1c1ccc(Cl)s1', 'O=C(C1CC[NH+](CCC(=O)N2CCNC[C@H]2CO)C1)N1CCN(c2ccccc2[N+](=O)[O-])CC1', 'O=C(c1cccc(NC(=O)N2CCN(c3ccc(F)cc3)CC2)c1)N1CCN(c2ccc(Cl)cc2)CC1', 'OC[C@@]1(COc2cccc(F)c2)CO[C@@H]2[C@@H]1[C@@H]1[C@H](C1)CN2CC1CC1', 'Clc1ccc(Br)cc1-c1cn(-c2ccccc2)nc1-c1cccc2nccnc12', 'Oc1ccc(F)cc1[C@H]1[C@H]([NH2+][C@H]2CCOc3ccc(Cl)cc32)OC1(CCc2ccccc2)c1ccc(F)cc1', 'Clc1ccc(-c2ccccc2)cc1Nc1ccc2nnc(-c3ccccc3)n2c1', 'Clc1cc(Br)c(CCNC(=O)[C@@H](Cc2ccccc2)[NH+]2CCN(c3ccccc3F)CC2)nc1C1CC1', 'Clc1ccccc1[C@H](c1oc2ccccc2c1[N+](=O)[O-])N1CC[NH+](Cc2ccccc2)CC1', 'Clc1ccsc1[C@H](N[C@H]1CCOc2ccccc21)[C@H]1CCCC[C@@H]1C(=O)N1CCCC1', 'Clc1ccc(C[NH+](Cc2ccccc2)CN2CCN(c3ccccc3C(F)(F)F)CC2)cc1', 'Oc1ccccc1/C(=N\\Nc1cccc(Cl)c1[N+](=O)[O-])C1(c2ccc(Br)cc2)CCCC1', 'Oc1cccc(-n2c(CCNc3ncnc4ccccc34)nc3ccccc32)c1', 'Clc1ccccc1[C@H]1[C@@H](Cc2ccccc2)c(=O)n(-c2ccccc2)N1c1ccc2c(c1)OCO2', 'Clc1cccc(/N=C/C2=CC(=O)CCC2=O)c1CN1CCC[C@@H]1CN1CCc2ccccc21', 'O=C(c1cn[nH]c1N1CCN(c2ccccc2C(=O)N2CCCC2)CC1)N1CCC[C@H]1c1ccccc1', 'Oc1ccc(/C=C2/C(=O)N(c3ccccc3)C(=O)N(c3ccc(Cl)cc3)c3ccc(F)cc32)cc1', 'Clc1ccccc1[C@H](Cl)[C@@H]1CN(C2CC2)CCO1', 'O=C(CN(C[C@H](O)c1ccccc1)C1CCCOCC1)N1C[C@@H]2C[NH+](CCCNC1=O)C2=O', 'Clc1ccccc1[C@H]([NH2+][C@@H]1CC[C@@]1(O)COc1ccccc1)c1ccc(Br)o1', 'O=C(c1c[nH]c2ccccc12)N1CCN(CC2CC[NH+](Cc3ccccc3)CC2)C1', 'O=C(C1CC[NH+](Cc2ccccc2F)CC1)N1CCc2ccccc2[C@H]1c1ccccc1', 'O=C(CCOn1c2ccccc2c2ccccc21)Nc1ccc2c(c1)CCCC2', 'O=[N+]([O-])c1c(Cl)nn(CC(=O)N2CCO[C@@H](Cn3ccc(Br)c3)C2)c1C1CC1', 'Clc1ccccc1COCC(=O)N1C2CC3CC(C2)CC1C3', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(Br)c1N1c2ccccc2CC[C@@H]1Cc1c[nH]c2ccccc12', 'Clc1ccccc1[C@H](N[C@H](c1ccccc1)C1CC1)C1CCCC1', 'Clc1ccccc1Oc1cc(F)c(OCc2nc(-c3ccc(Cl)cc3)[nH]n2)cc1', 'Clc1ccccc1[C@H](Cl)c1ccccc1OCc1ccccc1Cl', 'Clc1ccccc1Nc1cn(-c2ccccc2)nc1-c1ccccc1', 'O=C(C1=C([O-])C(=O)N(C[C@@H]2CCCO2)[C@@H]1c1cccc(Br)c1)c1ccc2c(c1)OCO2', 'O=C(c1cc(Br)cn1CC(F)(F)F)N1CCC[C@H](C(N2CCCCC2)c2ccccc2F)C1', 'Oc1cc(-n2nnc(-c3ccccc3)n2)c2ccccc2c1OCC[C@@H]1CCCC[NH+]1CC1CC1', 'Clc1cc(Br)cnc1-c1ccc(F)cc1OCC(=O)N1CCO[C@H](CCC(F)(F)F)C1', 'Clc1cccc(Br)c1COC1CC[NH+](CC2=c3ccccc3=[NH+][C@@H]2c2ccccc2)CC1', 'N=C1/C(=C/c2c(OCc3ccccc3)sc3ccccc23)C(=O)N1c1ccccc1', 'O=C(COc1cc(N2CCCCC2)ncc1O)N1CCN(C(=O)c2cc(F)ccc2Br)CC1', 'O=C([C@H](c1c(F)cccc1Cl)[NH+]1CCN(c2cccc(Cl)c2Cl)CC1)C1CCCC1', 'O=C(COc1ccc2ncccc2c1)Nc1ccc2nc(-c3ccc(Cl)s3)[nH]c(=O)c2c1', 'Clc1ccc(Br)cc1-c1cc(F)ccc1Cc1ccc(Br)cc1', 'Clc1ccccc1CC[C@H]1CN(C[C@@H]2CCCCO2)CCO1', 'Clc1ccccc1-c1nc(-c2ccccc2)sc1C(=O)N/N=C/c1ccccc1', 'O=C(C[NH+]1CC[C@@H]2CCCC[C@@H]21)NC[C@H]1-c2ccccc2CN1C(=O)c1ccc(F)cc1', 'O=C(C[NH+]1CCN(c2ccccc2)CC1)N1CCN(Cc2ccc(F)cc2)CC1', 'Clc1ccc(-c2nnnn2[C@@H](c2ccccc2)c2ccccc2F)cc1', 'O=C(CCc1nnc2ccccn12)Nc1c2[nH]c(=O)c3ccccc3c2c(Cl)cc1Cl', 'Clc1ccc(Br)cc1NCC(=O)N1CCC[C@@H](c2[nH+]ccn2CC2CCCC2)C1', 'O=[N+]([O-])c1cc([C@H]2CC(=O)N(c3ccccc3)c3ccccc3C2)ccc1N1CCOCC1', 'Clc1ccc(C(=O)NCCCN2CCO[C@@H](c3ccccc3)C2)cc1', 'Clc1ccc(-c2cc(-c3ccccc3O)nnc2-c2ccccc2)cc1', 'O=C(C[NH+]1CC[C@@H](c2nc3ccccc3s2)C1)NCCOc1ccc(N2C(=O)NC(=O)c3ccccc32)cc1', 'O=C([C@H](c1ccccc1)N1CCC(OC2CCCCC2)C1)N1CCN(c2ccccc2)CC1', 'Clc1ccccc1OC1(Cl)CCN(C(=O)c2ccccc2I)CC1', 'Clc1cc(CSC[C@@H](CSc2ccccc2)c2ccc(Br)cc2)c(F)c2c1CCCC2', 'Oc1ccc([N+](=O)C2C[C@@H](C3CCN(c4ccccc4F)[C@@H]3c3ccccc3)c3ccccc32)cc1', 'Clc1ccc(-c2cc[nH]n2)c(CCC(=O)N2[C@H]3CCn4c(nc5ccccc54)[C@@H]3C2Cc2ccccc2)n1', 'Clc1cc(Br)c(CN2CCO[C@@H](c3ccccc3)C2)cc1Br', 'Clc1ccc(Br)c(Cl)c1CCOCC1CCN(C(=O)[C@@H]2CC=CCC2)CC1', 'O=C(c1c[nH]nc1-c1cccs1)N1CCC2(CC1)NCCN2C(=O)C[C@@H]1C(=O)N(Cc2ccccc2)CN1Cc1ccccc1', 'O=C(/C(c1cccs1)c1ccccc1)[C@@H](c1ccccc1O)c1ccccc1', 'Clc1ccc(CCC2C=[NH+][C@]3(CCO/C(=C/C(=O)c4ccc5c(c4)OCO5)C3)[C@H]2C)cc1', 'Clc1cc(Br)c2nc(OCC(=O)N3CCN(C(=O)Cn4ccc5ccccc54)CC3)ncc2c1', 'Clc1ccc(-c2csc3[nH]cc(-c4noc5ccccc45)c23)cc1', 'O=C(c1c[nH]nc1CN1C[C@H]2CCC[C@]21[NH2+]C1CCCCC1)N1CCC(C(=O)N2CCCCCC2)CC1', 'O=[N+]([O-])c1c(OCOc2ccc(Cl)cc2)ncnc1N[C@H](c1ccccc1)c1ccc(Cl)cc1', 'O=C(Cc1c[nH]c2ccccc12)N1CCN(C(=O)c2ccccc2)CC1', 'Clc1ccccc1C(=O)NCC(=O)N1CCO[C@@H](c2ccc(F)cc2F)C1', 'Clc1ccc(-c2csc3s/cnc23)n2c1nc1ccccc12', 'O=C(c1c[nH]nc1C[NH+]1CCN(C(=O)[C@@H]2CCN(c3ccccc3F)CC2)C1)N1CCCCC1', 'O=C([C@H]1c2ccccc2C(=O)N(c2c(O)cccc2Cl)C1=O)N1CCCCN1CC1=CCCCC1', 'Clc1ccsc1SC[C@@H]1CN(Cc2ccc(Cl)cc2)CCO1', 'Clc1ccccc1-c1cscc1-n1ccc2ccccc21', 'N#C[C@@H](c1ccccc1)[C@H]1CCC[NH+](C2CN(C(=O)Cc3cnn(C4CC4)c3)C2)C1', 'O=C(c1c[nH]c2ccccc12)N(C[C@@H](O)c1ccc(Br)cc1)C1CCC1', 'Oc1ccc(O[C@H]2C=[NH+]C3=C(C(=O)c4ccc(Cl)cc43)[C@H]2/C=C/OCc2ccccc2)cc1OCC(F)(F)F', 'Oc1ccccc1[C@@H]1Cc2c([nH]c3ccccc32)c2ccccc21', 'OC[C@@]1(Cn2cccc2)CN(CCc2[nH+]ccn2Cc2ccc(F)cc2)C1', 'O=C([C@H](CNC(=O)c1ccc(N2CCOCC2)cc1)c1ccc(F)cc1)NNC(=O)c1ccoc1', 'Oc1ccc([N+](=O)c2oc3c(c2[C@@H](NCc2ccccc2)C[C@H](c2ccc(F)cc2)N3)c2ccc(F)cc2)cn1', 'Oc1ccccc1[C@H]1OCCN1CC[NH+]1CCCCC1', 'O=C([C@H]1c2ccccc2CCN1C(=O)C1(c2ccccc2)CCC1)NCc1ccco1', 'Clc1ccc(-c2ccccc2N2CCO[C@@H](CO)C2)cc1', 'O=C(c1c[nH]nc1N1Cc2ccccc2O[C@H](c2ccc(Cl)cc2)C1)N1CCC(c2nccs2)CC1', 'O=C([C@H](N[C@@H]1CCc2ccccc21)[NH+]1CCCC1)c1ccc2c(c1)OCCO2', 'COc1nn[nH]c1CCC(=O)N1CCC[C@@H](C(=O)NCc2ccccc2F)C1', 'N=C1\\C(=C\\c2ccc(C#CC[NH+]=C/c3ccccc3)cc2)NC[C@@]1(O)Cc1ccccc1', 'O=C(CNc1c(-c2ccc(Cl)cc2)cnn1-c1ccccc1)NNC(=O)c1ccco1', 'O=[N+]([O-])c1c(Cl)nnc2cc(-c3ccccc3)n(-c3ccc(F)cc3)c12', 'O=C([C@H](Nc1ccc2c(c1)OCCO2)c1ccccc1)N(C1CCCCC1)[C@H]1CS(=O)(=O)CCS1', 'O=C([C@H](c1ccc(F)cc1)N1CCN(c2ccc(Cl)cc2)CC1)c1ccc2ccccc2c1', 'O=C(c1cccn1C1CCCC1)N1CCOc2ccccc2[C@@H]1c1cc(F)cc(F)c1', 'C=C[C@@H]1N[C@@](CN2C(=O)C[C@@H](NCc3ccc[nH]3)N2)c2ccccc21', 'Clc1ccccc1Oc1cc(Oc2ccccc2)nnc1-c1ccccc1', 'O=C(c1cccn1-c1ccc(Br)cc1)N1CCC[C@H]1CC(=O)N1CCCC1', 'Clc1ccc([C@@H](OC[C@@H]2CCCC[NH2+]2)c2c[nH]c3ccccc23)cc1', 'O=C(c1ccsc1C1CC1)N1CCN(c2cnn(-c3ccccc3)c(=O)c2NCc2ccccc2F)CC1', 'Clc1ccc(-c2n[nH]c([C@H](c3c[nH]c4ccccc34)N3CCO[C@@H](C(F)(F)F)C3)c2)cc1', 'Clc1ccccc1CN(C[C@@H]1CC2(COCCO2)C1)[C@@H](c1ccccc1)C1CCCC1', 'Clc1ccc(-c2cc(Nc3ccccc3Cl)nc(-c3ccc(Cl)cc3)n2)cc1', 'O=C(c1cn(-c2ccc(F)cc2Cl)nc1NC[C@H]1CN2CCCC[C@@H]2CO1)Nc1ccc(F)cc1', 'O=C([C@H](C(Cl)F)N1CCN(CC(=O)N2CCOCC2)CC1)Nc1ccccc1', 'O=C(/N=c1/[nH]c[nH+]c2ccccc12)c1ccc2c(c1)C(=O)N(C1CCCC1)C2', 'Clc1ccccc1[C@H](C[NH+](Cc1ccccc1)C1CCCC1)Nc1cc(F)ccc1Br', 'O=C(COCCCOc1ccccc1)NCCC12CC3CC(CC(C3)C1)C2', 'Clc1ccccc1Nc1nnnn1-c1cccc2ccccc12', 'Oc1ccc(Oc2nc(-c3ccccc3)oc2-c2ccccc2)cc1Cl', 'Oc1ccccc1[C@H](NCc1nc(-c2ccccc2)n[nH]1)c1ccc(Cl)cc1', 'O=C(CCCOc1ccccc1Br)N1CCCCCCC1', 'O=C(Cc1n[nH]nc1C1CC1)NCC1CC[NH+](CC2(c3ccccc3F)CCCC2)CC1', 'Clc1ccccc1Oc1cc(OCc2ccccc2)nc(-c2ccccc2)n1', 'Clc1ccccc1-c1n[nH]cc1COc1ccc(-c2ccccc2)nc1', 'Clc1ccccc1NCc1cc[nH+]cc1-[nH+]1cc[nH+]c1', 'N[C@@H]1c2ccccc2O[C@@H]1C(=O)N/N=C/[C@@H]1CCCC2=Cc3ccccc3N21', 'Oc1ccc(Oc2ccc(/C=N/NC[C@@H](c3ccccc3)c3ccco3)nc2Cl)cc1', 'O=C(COCCNc1ccc(Br)cc1)N1CCN(c2cccc[nH+]2)CC1', 'O=C(C[NH+]1CCC[C@H]1c1ccc(Cl)cc1)N1CCC[C@@H]1c1ccccc1', 'Clc1ccccc1CNc1nc2ccc(C(F)(F)F)cc2n1CC[NH+]1CCC(Cc2ccccc2)CC1', 'Clc1ccccc1[C@H]1CCC[NH+]1CCCN1CCN(c2cccc[nH+]2)CC1', 'Clc1ccc(COc2nc3ccccc3c3sccc23)c2ccccc12', 'Clc1ccccc1C1=C[C@@H]2[C@@H](CCCc3ccccc31)[C@@H]1CCC[C@@H]12', 'O=C(c1cccn1Cc1ccccc1)N1CCN(c2ccc[nH+]c2)CC1', 'Clc1ccccc1Oc1cnc(Cc2ccccc2)n(-c2ccccc2)c1=O', 'Oc1ccc(Oc2cncc(-c3ccccc3COc3ccccc3)c2)cc1', 'O=C(C1CCN(c2ccc([N+](=O)[O-])c(F)c2)CC1)N1CCC[C@H]1CC(=O)Nc1ccccc1Br', 'Clc1nnnn1c1[nH]ncc1[C@@H](c1ccccc1)N1C(=O)c2ccccc2C1=O', 'O=C(/C(c1cccs1)=[NH+]/C[C@H]1CC=Cc2sccc21)N1CCN(c2ccccc2)CC1', 'Clc1ccccc1OCc1ccc(-c2cccc3ccccc23)o1', 'O=[N+]([O-])c1cc(Br)c(C(=O)N2CCN(c3ccccc3[N+](=O)[O-])CC2)cc1OC(F)F', 'O=C(CNCc1csc2ccccc12)N1CC2(CCCCC2)Oc2ccccc21', 'O=C([C@H](c1ccc(Br)cc1)N1CCN(Cc2c[nH]c[nH+]2)CC1)c1ccccc1F', 'Clc1ccc([C@@H](Cl)C[C@H](Cc2ccccc2)N2CCOCC2)cc1', 'Oc1cc(OC2=NN(c3ncccn3)C[C@@H]2c2ccccc2)ccc1OCCOc1ccccc1F', 'O=C(c1c[nH]c2ncn([C@H](CO)c3nc4ccccc4cc3)c12)N1CCN(C(=O)c2ccccc2)CC1', 'ClC[C@H]1CN(Cc2cccc(O)c2)CCO1', 'O=C(c1cn[nH]c1NC1CCN(C(=O)COc2ccccc2)CC1)N1CCO[C@@H](C2CC2)C1', 'Clc1ccc(Br)c(S[C@H](C2CCCCC2)C2CC2)c1', 'Oc1ccc(/C=N/Nc2ncnc(Nc3ccccc3)c2N2CCCC2)c2c1CCCC2', 'Clc1ccccc1OC1(O)CCN(c2cccc[nH+]2)CC1', 'O=C(CNCc1nonc1C1CC1)N1CCC[C@H](c2nccn2C2CC2)C1', 'Oc1ccccc1CCC1CCN(Cc2ccccc2-n2cc[nH+]c2C)CC1', 'Clc1ccccc1[C@H]([NH2+]C1CCCCC1)[C@H]1CO[C@@H]2C[C@@H]1c1ccccc1C2', 'O=C(c1cn2ccsc2c1NC[C@H](O)c1ccc(Br)cc1)N1CCC(O)CC1', 'O=C(CNc1cnccn1)N(c1ccc2ccccc2n1)c1ccccc1', 'Oc1ccccc1COc1ccccc1C(=O)NC[C@H]1CCCc2ccccc21', 'O=C(CN(C[C@H]1CCCO1)c1ccccc1)N[C@H](c1ccccc1)c1ccccc1', 'O=C([C@H]1COCCN1c1c(F)cccc1F)N1CCN(C(=O)COc2ccccc2)CC1', 'Clc1ccccc1OCC(=O)N1CCC[C@@H](CCc2ccccc2)C1', 'O=C(/N=c1\\sc2ccccc2c(=O)n1-c1ccccc1)C1CCN(C(=O)c2ccc(Cl)s2)CC1', 'O=C(CNc1nncc2c1nc(N)n2C)N1CCN(C2CCCCC2)CC1', 'O=C(Cc1nn(CN2CCCCC2)c2c1CCC2)N1CCC[C@H](Oc2ccccc2Cl)C1', 'Clc1ccc(CN[C@@H](Cc2c[nH]c3ccccc23)C2CC2)cc1', 'O=C(C[NH+]1CCCN(c2cc(F)cc(F)c2)CC1)Nc1ccccc1C(=O)NCc1ccccc1', 'O=C(C1=C([O-])C(=O)N(Cc2c[nH]c3ccccc23)C1=O)N1[C@@H]2CC[C@@H]1Cc1ccccc1-2', 'O=[N+]([O-])c1c(-c2noc(-c3ccc(F)cc3)n2)nnn1-c1ccccc1', 'Oc1ccccc1[C@H]1CN(c2ccccc2)CCN1CCn1cc[nH+]c1', 'O=C(Cc1coc2ccccc12)Nc1ccccc1-c1ccccc1', 'O=C(Cc1c(NC2CC2)c[nH]c(=O)c1Cl)NC[C@H]1CCc2ccccc21', 'Clc1ccccc1[C@H]1[C@@H](NC(=O)c2ccc3c(c2)CCCC3)CC[NH+]1Cc1ccccc1', 'O=C(c1c[nH]c(NN=C(c2ccccc2)c2ccccc2)n1)NNc1ccc(F)cc1', 'Oc1ccc(F)c(SCC(=O)N2CCO[C@@H](c3ccc(Cl)cc3)C2)c1', 'Clc1ccc(C[NH+](CC(=O)N2CCC[C@@H]2CC2CCCCC2)C2CC2)cc1', 'Clc1ccc(-c2nonc2Cn2cnc(-c3ccc(Cl)cc3)n2)cc1', 'Clc1cccc2[nH]nc(-c3ncc(-c4ccccc4)nc3)c12', 'O=C(/N=c1\\sc(C(Cl)(Cl)F)nn(CN2CCCC2)c1=O)c1cc(Br)sc1Cl', 'O=[N+]([O-])c1cc(F)ccc1N1CCC([NH+]2CCCc3cc(F)ccc32)Cc2ccccc21', 'O=C(c1ccc(NC(=O)c2sc(N3CCOCC3)nc2C2CC2)cc1)N1CCC[C@@H]1c1ccc(Cl)cc1', 'Clc1cc(N2CCOCC2)nc(C[NH+]2CCC[C@]3(CCC(=O)N(Cc4ccccc4)C3)C2)c1', 'Clc1ccccc1OCCN(C(=O)Nc1cccc2[nH]ccc12)C1CC1', 'Oc1cc(C=N[C@@H]2CC[NH+]=C2c2ccco2)ccc1O', 'O=C(COc1ccccc1Cl)N[C@H](Cn1cc[nH+]c1)c1ccccc1', 'O=C(/C=C1/N(Cc2ccccc2)c2ccccc2C1=O)Nc1ccccc1OC1CCCC1', 'Clc1ccccc1C1=CO[C@H]2C[C@@H](CC(=O)OCc3cccc(Cl)c3)N12', 'Clc1cc(Br)c2nc([C@H](c3ccc(F)cc3)[NH2+][C@H](c3ccco3)N3CCCCCC3)sc2c1', 'O=C(c1cc([N+](=O)[O-])ccc1N1CCN(c2ccccc2)CC1)N1CCC(OC2CCOCC2)CC1', 'O=C(c1cccn2cc(-c3ccccc3)nc12)Nc1ccc(Cl)cc1', 'Clc1cccc(Nc2cnc(-c3cnn(-c4nc(C(F)(F)F)ccc4Cl)c3)[nH]c2=O)c1', 'O=[N+]([O-])c1cc(C(=O)NC[C@H]2CCC[NH2+]C2)[nH]c1N1CC=C(c2ccc(F)cc2)CC1', 'Clc1ccsc1CNC[C@@H]1OCCN(C(=O)c2cccs2)C1', 'Clc1ccsc1/CCN(C(=O)c1ccc(N(C2CC2)c2ccccc2)cc1)C1CC1', 'Clc1ccccc1Nc1c2on(-c3ccccc3)c(=O)c2cc2c1CCCCC2', 'Clc1ccc(Br)cc1N(C[C@@H]1CCCO1)[C@H]1C[C@@H]1c1cc(Br)ccc1O', 'Clc1ccc([C@@H](NC[C@H](O)c2ccc[nH]2)c2ccccc2)cc1', 'C[C@@H](c1ccc[nH]1)N(C)C(=O)[C@@H]1CC(=O)N(Cc2ccccc2F)C1', 'ClC[C@H](c1c[nH]c2ccccc12)N1c2ccccc2C[C@@H]1Cc1cccs1', 'Clc1ccccc1N1N(CCn2cc[nH+]c2-c2ccco2)CC1', 'O=C(C[NH+]1CC[C@@H]2[C@@H](c3ccccc3)[C@H]2C1=O)NCc1ccccc1', 'O=[N+]([O-])c1ccnc2c1c(=O)n(Cc1cccnc1)[C@H](c1ccccc1Cl)c1ccccc1N2', 'Clc1cccc(Br)c1NC[C@@H]1CCCc2ccccc21', 'O=C(/C=C/c1ccc(Br)cc1)N(C[C@H]1CCCO1)c1ccccc1', 'Clc1ccccc1OCCC(=O)N1CCC[C@H](C(=O)N2CCCC2)C1', 'Oc1cc(OCC(=O)N2CCCCC[C@@H]2c2ccc3c(c2)OCO3)cc(OCC(F)F)c1', 'Clc1ccccc1NC1CCN(c2ccccc2OCc2ccc(F)cc2)CC1', 'O=C([C@H](CCO)Nc1cc(F)ccc1F)NC12CC3CC(CC(C3)C1)C2', 'O=C(/C(COc1ccccc1)c1ccccc1)[C@@H]1COc2ccccc2O1', 'Clc1ccc(-c2n[nH]c(CN(Cc3ccco3)C2CCCCCCCOCCOCCO)c2)ccn12', 'O=C(c1ccc(OCC(=O)N2CCCCC[C@@H]2C(=O)[O-])cc1)N1CCN(c2ncccn2)CC1', 'Clc1ccc(-c2nnnn2[C@@H](c2cccs2)c2ccccc2)cc1', 'O=C(c1ccc(-n2cncn2)cc1)N1CCN(c2cc(-c3ccccc3)ncn2)CC1', 'Oc1ccc(F)c(/N=C\\Nc2ccccc2F)c1NS(=O)(=O)c1ccccc1', 'Nc1nnc(C2=NN2c2cccc(C(F)(F)F)c2)n1/N=C/[C@H]1CCCC[C@@H]1C(=O)Nc1cccc2[nH]ccc12', 'Clc1ccc(Br)c(N2CCN([C@@H]3CCCC[NH+]3Cc3ccccc3F)CC2)c1', 'O=C(/C=C/c1c(-c2ccccc2Cl)noc1-c1cccnc1)Nc1cccc(C(=O)[O-])c1', 'O=C(c1c[nH]c([O-])n1)N(Cc1ccccc1)C[C@H](c1ccccc1)C(=O)Nc1ccc2c(c1)OCCO2', 'Clc1ccccc1O/C=C(c1ccccc1)N1CCN(c2ccccc2)CC1', 'Clc1cc(Cl)c(CCCOc2ccccc2Cl)c(Br)c1Cl', 'O=C(/N=c1/oc(Cc2ccccc2)cc2c1N[C@H](c1ccco1)S2)c1ccc(F)cc1', 'Clc1ccc(-c2nnnn2-c2cc(Nc3ccccc3)ccn2)cc1', 'O=C(c1c[nH]nc1N1CCOCC1)N1CCN(C[C@H]2CCCC[NH+]2Cn2cncn2)CC1', 'O=C(CNC(=O)C1CCCC1)N1N=C(c2ccco2)C(=O)[C@@H]1C(c1ccc(F)cc1)c1cccs1', 'Clc1ccc([C@@H](N[C@@H]2CCc3ccccc32)c2ccccc2)cc1', 'Oc1ccccc1COC1CCN(C(=O)Nc2ccccc2Oc2ccc(F)cc2)CC1', 'O=C(c1cn(-c2ccccc2)nc1-c1ccccc1)N1CCN(C2CC2)CC1', 'O=C(/C=C1/cc[nH]1)NCC1(N(C)C(=O)c2ccccc2)CCCCC1', 'O=C([C@H](c1ccco1)N1CCC(c2[nH]cc[nH+]2)CC1)c1ccccc1', 'N#Cc1c(-c2ccccc2)cc(C(=O)N[C@@H]2CC(=O)N(Cc3ccccc3)C2)n1C1CC1', 'Oc1ccccc1-c1nnn(CCc2ccc(F)cc2)c1-c1ccccc1', 'Oc1cc(-n2nnc(/N=C/c3ccccc3)c2-n2cncc2C(F)(F)F)ccc1F', 'COc1nn[nH]c1CC(=O)N1CC[C@@H](C(=O)[O-])[C@@H]1C', 'Clc1cccc(Br)c1NC[C@H]1CC[C@H](c2ccccc2)[NH2+]1', 'N=Cc1ccc(O[C@H](C)C(=O)N2c3ccccc3C[C@H]2c2ccccc2)cc1', 'O=C(COc1cc(Cl)ccc1I)Nc1nc2c(s1)CO[C@@H](c1ccccc1)C2', 'Clc1ccccc1Oc1cc(Br)c(COc2ccc(Br)cc2)nc1N1CCOCC1', 'Clc1ccccc1[C@H](c1ccccc1)n1c([O-])nnc2ncnc2c1=O', 'O=C(CS(=O)(=O)C[C@@H]1CN(Cc2ccccc2)CCO1)Nc1cccc2ccccc12', 'O=[N+]([O-])c1ccn([C@H](CO/N=C\\c2c[nH]c3ccccc23)N2CCOCC2)c1', 'O=C(CN(c1cnn(C(C)C)n1)c1ccccc1)N1CCN(C2CC2)CC1', 'O=C(c1cc([S@](=O)c2cccs2)ccc1N1CC[NH+](CC(=O)N2CCOCC2)CC1)[C@H]1CCOC1', 'O=C(c1cn(-c2ccccc2)nc1-c1cnn(-c2ccccc2)c1)C1CCOCC1', 'Clc1ccc(-c2cc(-n3cccn3)nc(-c3cc(F)ccc3F)n2)cc1', 'ClC(c1ccccc1)C1(C(=O)NC2CCCC2)CC(CC2CCCC2)C1', 'Oc1cccc2nc([C@H](c3ccccc3)N3CC[NH+](C[C@H]4CCCO4)CC3)ncc12', 'Clc1ccc([C@@H](NNC(=O)c2ccc(N3CCCCC3=O)cc2)N2CCCCC2)cc1', 'O=C(Cc1c[nH]c2ccccc12)NC1CCN(c2cccc[nH+]2)CC1', 'O=C(Cc1nn(CC[NH+]2CCCCC2)c(=O)c2ccccc12)N1CCOCC1', 'ClC(c1cccs1)c1cc(CC2CCN(C(=O)c3ccc(F)cc3)CC2)nc2ccccc12', 'COc1nn[nH]c1CC1CC[NH+]([C@@H](C(=O)N2CCCCC2)c2ccccc2)CC1', 'ClCc1csc(N2CCO[C@@H](c3ccccc3)C2)c1Cl', 'Clc1cccc2c1[C@H]1C[C@@H]1C[NH+]2Cc1ccccc1C[N+]1CC1', 'Clc1ccccc1-c1cscc1-c1ccccc1C(=O)N[C@@H]1CCCc2ccccc21', 'Clc1cc(/C=C2\\C(=O)N=C(Nc3nc(-c4ccccc4)cs3)C[C@@H]2c2ccc(F)cc2)n[nH]1', 'Clc1ccccc1Cc1nccc2ccc(C(=O)Nc3cnn([C@@H]4CCC[NH+](C)C4)c3)cc12', 'CN[C@H](NC(=O)NC[C@H](O)[C@@H]1CCCO1)c1ccccc1', 'O=C(/C=C\\C1C2CC(CC(C2)C1)[C@@H](CO)c1ccccc1O)N1CCc2ccccc21', 'O=C(c1cncc(Cl)c1)C1CCN(c2cc(-c3ccccc3)n[nH+]c2-c2cccs2)CC1', 'Clc1ccccc1Cn1cnc2ccccc21', 'O=C([C@H]1[C@H]2C=C[C@@H](CN2c2ccccc2)C1)N1CCC(Cn2ccnc2)CC1', 'O=[N+]([O-])c1c(Br)sc(Br)c1/C=N/OCCn1ccc2ccccc21', 'Clc1ccccc1Oc1cc(-c2n[nH]c(COc3ccccc3)n2)c[nH]1', 'Oc1ccccc1[C@H](C[NH2+][C@@H](CO)c1ccc(Cl)cc1)c1ccccc1', 'O=C(/C=C/c1ccnc2c1-c1ccccc1O2)N1CCC(Cc2ccccc2)CC1', 'Clc1ccccc1OCc1ccc(NC[C@@H]2COc3ccccc3O2)cc1', 'Clc1cccc(Nc2ccc(Cl)c(C(=O)Nc3nnc(-c4ccccc4)o3)n2)c1', 'Oc1ccccc1-c1ccc(=O)n(Cc2nc(-c3cccc(F)c3)no2)c1', 'Clc1cccc2c1N[C@H](c1ccc(NCc3ccc(Cl)cc3)[nH]c1=O)CC(O)(CO)C2', 'Clc1ccccc1OC1CC[NH+](CC[C@@H]2CCN(C3CC3)C2)CC1', 'Clc1ccccc1CCn1cc[nH+]c(C[NH2+][C@H](CO)c2ccccc2C(F)(F)F)c1=O', 'O=C(c1c[nH]c2ncn(C(=O)c3ccccc3)c12)N1C[C@@H](COc2ccccc2F)C1', 'Clc1ccc(C(F)(F)F)n1NCC1CCN(C(=O)CC2=COCCO2)CC1', 'O=C(c1cnc(N2CCc3ccccc32)nc1)N1CC[C@@H]([NH+]2CCCC2)C1', 'O=C(c1c[nH]c2cccc(F)c12)N1CCC[NH+](Cc2ccccc2)CC1', 'O=C(C1CC[NH+](C2CCCC2)[C@@H](C(=O)NC2CCCCCCCCC2)C1)c1ccccc1', 'Clc1cccc(Br)c1N1C[C@@H]2[C@@H](C1)CN(Cc1ccc(Br)cc1)N2', 'Clc1cc(/C=C(\\CCl)C2CC[NH+](Cc3ccccc3F)CC2)cc2[nH]ccc12', 'O=[N+]([O-])c1ccnc2nc(C3(c4ccc(Br)cc4Cl)CCC3)n(Cc3ccccc3)c(=O)c12', 'Clc1ccccc1Nc1c2ccc(Oc3ccccc3)nn2c(=O)cn1', 'Clc1ccccc1Oc1cc([N+](=O)Nc2ccccc2)ccc1N1CCN(C(=O)c2ccccc2)CC1', 'Clc1cccc(Br)c1CSCCCCC[NH+]1CCCCC1', 'Clc1ccccc1CN(CCCn1cc[nH+]c1)C[C@H]1CCCO1', 'Clc1ccc(C(C)C)cc1CN/C(=C/c1ccccc1)c1ccccc1', 'Clc1ccccc1NC1CCN(C(=O)c2ccc(OCc3ccccc3)s2)CC1', 'Clc1cccc2c1NC[C@H](c1ccccc1)CN2Cc1ccccc1', 'O=C(Cc1cnc(-c2ccncc2)c(Cl)c1)NC(c1ccccc1)c1ccccc1', 'O=C(CNc1nn2c(=O)cc(-c3ccc(Cl)cc3Cl)cc2s1)Nc1cccc2ncccc12', 'O=C(c1ccsc1)N(c1ccc2ccccc2c1)[C@@H]1CCO[C@@H]1c1ccc(Cl)cc1F', 'O=C(Cc1c[nH]nc1-c1ccccc1)[C@H]1CCC[NH+](Cc2c[nH]c3ccccc23)C1', 'Clc1ccccc1Oc1cc(C[NH+]2CCC(Cc3nc4ccccc4[nH]3)CCCC2)ccc1O', 'Oc1cccc([C@H]2OCCN2C(=O)CSCc2ccccc2)c1', 'Clc1ccc(-c2nn([C@@H](Cc3ccccc3)c3ccc(F)cc3F)nn2)cc1', 'Clc1cccc(F)c1C1CC[NH+]([C@@H]2CCN(c3ccccc3)C2)CC1', 'Clc1cc(Nc2c(cnn2Cc2ccco2)c2ccccc2)ccc1[N+](=O)[O-]', 'Clc1ccc(-c2ccccc2COC2CCN(c3nc4ccccc4o3)CC2)cc1', 'O=C(/N=C/NCCC[NH+]1CCCCCCC1)N[C@H]1CCCOc2ccccc21', 'Clc1ccc(CN2C(=O)NC3(CCN(CC(=O)N4CCOCC4)CC3)C2=O)cc1', 'O=C(c1c[nH]cn1)Oc1cc(CNc2ccccc2)n(-c2ccccc2)n1', 'COc1ccccc1C(=O)OCCC(=O)N1C[C@H](C)C[C@@H](C)C1', 'COC(C)(C)CN(C)C(=O)c1ccc[nH+]c1NC(=O)CC1CCC1', 'Clc1ccc(CCCN2CCc3[nH]c4ccccc4c3C2)cc1', 'O=C(c1c[nH]nc1CN1CC[NH+](Cc2ccccc2)CC1)N1Cc2ccccc2C1', 'Clc1ccccc1CNC1CCN(CN2CCc3ccccc32)CC1', 'N=CCNC(=O)C1CCN(C(=O)CNC(=O)c2ccc(F)cc2)CC1', 'O=C(/C=C/Oc1ccc(Cl)cc1Cl)NC[C@@H](Cc1c[nH]c2ccccc12)[NH+]1CCCCC1', 'Clc1ccc(Br)c(Cl)c1OCCN1CCN(c2cccc[nH+]2)CC1', 'Clc1ccc([C@@H](Br)[C@H](Cc2ccccc2)[NH+](C)C2CC2)cc1', 'Clc1ccccc1O/N=C1\\SC(=S)N(CC[NH2+]C2CCCCC2)[C@H]1c1cccc(Br)c1', 'Clc1ccc([C@@H](C[NH+]2COc3ccccc3OC2)c2ccccc2)cc1', 'O=C(c1cccn1CCO)N1CCC(OC2CCCCCC2)CC1', 'Clc1cccc2cc([C@H](Cl)c3ccc(OCc4ccccc4F)cc3)sc12', 'N=C1\\C[NH+](CCCCN2CCCCC2)[C@H](C)C(=O)N1c1ccccc1', 'Clc1cc(/C=C(/C#CCN2CCOCC2)c2ccccc2)ccc1Br', 'Clc1cccc(Br)c1ONCc1ccc[nH]1', 'Clc1ccc(C2SCCCN2CCSC2CC2)cc1', 'O=C(c1ccsc1COc1ccccc1)N1CCN(C(=O)COc2ccccc2)CC1', 'Clc1cc(Br)c(Br)c(CCNc2ncnc3c2cnn3Cc2ccccc2)c1', 'Oc1cc(-n2cnc(-c3csc(-n4ncn(CCO)c4=O)c3)c2-c2ccccc2)ccc1O', 'Clc1ccc(C(CCF)N2CCC[C@H](c3nc4ccccc4[nH]3)C2)cc1', 'O=[N+]([O-])c1cccc2c1[C@@H]1CCC[C@@H]2[C@@H]2C[C@H]([n+]3ccco3)CCC[C@@]21O', 'Clc1ccc(-c2cc(O[C@H](Cc3cccc(F)c3)c3ccccc3)n[nH]2)cc1', 'Clc1ccc([C@@H]([NH2+]Cc2ccccc2)C2CCN(C3CCCCC3)CC2)cc1', 'O=C(c1ccsc1)[C@@H]1CN(Cc2cccs2)CCO1', 'ClC[C@H](c1ccc(Br)s1)N1CCC2(CC1)COc1ccccc12', 'Clc1ccccc1Oc1cnc(-c2n[nH]c3ccccc23)cc1', 'Clc1ccccc1Oc1cnc(-n2cnc3ccccc32)nc1', 'Clc1cccc2nc(N3CCC[C@@H](CC4CC[NH+](C5CCCCC5)C4)CC3)nn12', 'Clc1ccccc1Oc1cc(Br)c(N2CC[NH+](CC(=O)NC(C)(C)C)CC2)cc1', 'O[C@@H]1[C@@H](CC(=O)N2CCN(Cc3ccccc3F)CC2)CC(=O)N1C[C@@H]1CCCCO1', 'Clc1cccc(Br)c1N[C@@H]1CCOC2(C[C@@H](COCc3ccccc3)C2)C1', 'O=C(COc1cc(C[NH+]2CCOCC2)ccc1Br)NC[C@H](c1ccccc1)[NH+]1CCCC1', 'COc1ncnc(NNC(=O)c2cc3ccccc3nc2-c2ccccc2)n1', 'Clc1ccccc1N1N(C2(NS(=O)(=O)c3ccccc3)COC2)SC1=O', 'Clc1ccccc1OCc1ccc(F)cc1NC(=O)c1ncn(Cc2ccccc2)n1', 'N=C1C[NH+](C2CC[NH+](CC(N)=O)CC2)CCN1Cc1ccccc1', 'C[C@@H](C[NH+]1CCN(CC(=O)N2CCCCC2)CC1)c1ccc(Cl)cc1', 'Oc1ccc(Oc2ccc(NCC(=O)N3CCCc4ccccc43)cc2)cc1', 'O=C(c1cn2ccccc2n1)N1CCN(Cc2ccccc2F)[C@H](C(=O)c2ccc(Cl)cc2)C1', 'Clc1ccc([C@@H](O)[C@@H]2CC(c3ccc(O)cc3)c3ccccc32)cc1', 'Clc1ccc(CCN[C@H]2CC(=S)N(C3CCCCCC3)C2)cc1', 'Clc1ccccc1Oc1cncc(/C=C/N2CCN(CCC[NH+]3CCCCCC3)CC2)c1', 'O=C(COc1cc(C2CC2)nc2ccccc12)N1CCCC[C@@H]1c1nc2ccccc2[nH]1', 'C=CCn1cc(C(=O)NNC(=O)N2CCN(c3ccc(Cl)cc3)CC2)c2ccccc21', 'O=C([C@H]1CCOC2(CCOCC2)C1)N1CCN(C(=O)[C@@H]2C[C@@H]2c2cccc(F)c2)CC1', 'Oc1cc(-n2cccn2)ccc1NC1=NC=CN=C1c1ccccc1Cl', 'O=C(c1c[nH]nc1C1CC1)N1CCC[C@H](N2CCN(C3(c4ccccc4)CCO3)CC2)C1', 'O=C(c1c[nH]nc1-c1ccco1)N1CCC[NH+](Cc2ccccc2)c2ccccc21', 'Clc1nc(-c2ccccc2F)nc2c1cnn2-c1ccc(Br)cc1', 'O=C([O-])/C(C[C@@H](O)c1ccccc1)N(Cc1nc2ccccc2n1Cc1ccccc1)C1CC1', 'O=C([C@H]1[C@H](c2cccc(Br)c2)C[C@@H]1c1ccccc1)N1CCCN(CC(=O)[O-])CC1', 'Clc1ccc([C@@H](NS[C@H]2CCOC3(CCCC3)C2)N2CCOCC2)cc1', 'O=C(/C(CNN=C1CCCC1)c1ccc(Cl)cc1)CC1CC[NH+](C2CCCCC2)CC1', 'Clc1ccc([C@@H]([O-])[C@@H](c2ccccc2)N2CC[NH+](CC(=O)NC3CCCC3)CC2)cc1', 'Clc1ccc(-c2nnnn2CCc2cnc3ccccc3n2)cc1', 'Clc1ccc([C@@H](NN[C@@H]2CCc3ccccc32)[NH+]2CCCC2)cc1', 'O=C(/C=C/Oc1ccccc1F)N[C@H]1[C@H](C[NH+]2CCCCC2)C[C@H]1c1ccc(F)cc1', 'O=C(C[NH+]1CCC([C@H](Br)C2CC2)C1)N[C@@H]1CCCc2ccccc21', 'O=C(c1c[nH]nc1CN(Cc1ccccc1)C1CC1)Nc1ccc2c(c1)OCO2', 'O=C(c1ccc[nH]c1=O)[C@@H]1CCC[NH+](Cc2ccc(Cl)cc2)C1', 'O=C(/C(c1ccccc1)C1=CCCCC1)[C@@H]1CC=CCC1', 'Clc1ccccc1[C@H]1[C@@H](n2cccn2)CCC[NH+](CCc2ccccc2)C1', 'O=C(CNCc1nc(-c2ccccn2)no1)N1CCN(C[C@@H]2CCCO2)CC1', 'O=C(Cc1nn(Cc2ccccc2)cc1COc1ccc(Br)cc1)c1ccccc1', 'Clc1ccc([C@@H](Br)C[C@H](c2ccc(Cl)cc2)[NH+](C)C)cc1', 'Clc1ccccc1N1N(C2CCCCCCC2)c2ccccc21', 'O=C(/C(c1cccs1)[NH+](C)Cc1ccc(Cl)cc1)C1(N2CCOCC2)CCCC1', 'Clc1ccc(-c2nn([C@@H](O)[C@@H](CC(=O)NC3CC3)C(C#N)=C3CCCCC3)n2Cc2ccccc2)cc1', 'O=C(/C=C/c1ccc(C(F)(F)F)cc1)OCCCCc1ccccc1', 'O=C([C@H](c1c(F)cccc1F)Oc1cccc(C(N)=O)c1)N1CCN(c2ccccc2F)CC1', 'Clc1ccc(Br)cc1CN(Cc1ccc(Cl)cc1Cl)C[C@@H](O)C1CC1', 'Clc1ccc(-c2ccccc2CCn2cnc3ccccc32)cc1', 'O=C(CNc1cnn(CN2CCCC2)c1)NCc1nc2ccccc2c2c1CCCC2', 'O=[N+]([O-])c1c(F)cc(N(C(=O)CN2CCOCC2)c2nc3ccccc3s2)c2ccccc12', 'Cc1ccccc1OCC(=O)N1CCO[C@@H](c2cccc(C(F)(F)F)c2)C1', 'O=C(CN1CNc2[nH]c[nH+]c2[C@@H]1c1ccccc1)NC[C@H](O)c1ccccc1', 'Clc1ccccc1CCCC(=O)Nc1ccccc1OCc1ccc(F)cc1', 'Clc1ccc(C(=O)O[C@@H]2C[C@@H](c3ccccc3)[NH+](Cc3ccccc3F)C2)cc1', 'O=C(c1ccc[nH]1)N1CC[C@@H](c2ccccc2)[C@@H]1C#CCc1ccccc1', 'Clc1ccccc1Oc1cnc2ncnc(-c3ccccc3)c2c1', 'Clc1ccccc1Nc1cccc(Cn2ccc3ccccc32)c1', 'Oc1ccccc1C1=C(N/N=C/c2ccccc2)C=C1c1cc[nH]c1', 'Oc1cc(-n2cc(F)cc2Cn2cc[nH+]c2)ccc1Nc1ccc(F)c(F)c1', 'Clc1ccccc1[C@H](NC1CC1)c1sccc1Br', 'O=C(CNCc1noc(Cc2ccccc2)n1)N1CCN(c2ccccc2F)CC1', 'Clc1ccc(-c2nn(-c3ccc(OCc4ccccc4)cc3)c(Cl)c2)cc1', 'O[C@@H]1c2ccccc2CN1Cc1ccc(-c2ccccc2)cc1', 'Oc1ccccc1CO[C@@H]1CCCC[C@@H]1[C@@H]1CCCN1C(=O)c1ccccc1', 'Clc1ccccc1[C@H](NCc1ccc(Br)cc1)c1ccccc1', 'Clc1ccc(C(CC)(NCC(F)(F)F)C(F)(F)F)cc1', 'Clc1ccc(-c2cc(/C([O-])=NO[C@@H]3CCC[C@@H](CO)[C@H]3NC(F)(F)F)nn2C2CCCCC2)cc1', 'O=C(Cc1nc2ccccc2n1Cc1ccccc1)Nc1ccc2c(c1)OCCCO2', 'O=C(C1CC[NH+]([C@@H]2CCN(C(=O)c3ccc4c(c3)OCCO4)CC2)[C@@H]1O)c1ccccc1', 'O=C([C@H](c1ccc(F)cc1)N1CCOCC1)N(CCO)Cc1c[nH]c2c(Cl)cccc12', 'O=C(c1c[nH]nc1N(c1ccccc1)c1cccc(F)c1)c1cccc(F)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(I)cc1N1[C@@H]2CC[C@H]1C[C@@H]2c1ccc(F)cc1', 'Clc1ccccc1Oc1cc(Cc2csc(-c3ccco3)n2)cnc1[O-]', 'Clc1ccc(Br)c(F)c1-c1ccccc1OCc1cc(F)cc(Cl)c1', 'CNc1nn(Cc2ccc(Cl)cc2)c(N2CCCC2)c1/C=C/c1ccc(O)cc1', 'Oc1ccc(O[C@H](C(=O)NNC(=O)c2nc(C3CC3)c3ccccn23)c2ccccc2)cc1', 'Clc1ccccc1Cc1cnc([C@@H]2CCO[C@H]2C(CC2CC2)c2ccccc2)c(Cl)c1', 'Clc1ccccc1CC1(C[NH2+][C@@H]2CCCc3ccccc32)CCCCCC1', 'Nc1c(Cl)cccc1NC(=O)[C@@H]1CC(=O)N(CCc2c[nH]c3ccccc23)C1', 'Clc1ccc(COc2nc(N3CCCCC3)[nH]c2N2CC[C@H](C[NH+]3CCCC3)C2)cc1', 'Clc1ccccc1NS(=O)(=O)N[C@]1(CO)CCC[C@@H]1Cc1ccccc1', 'O[C@@H]([C@@H](n1cc(C(F)(F)F)c2ccccc21)C1CCCC1)N1CCOCC1', 'Oc1ccccc1C1CCN(C(=O)CN2c3ccccc3CC2)[C@H]2CS(=O)(=O)C[C@@H]21', 'O=[N+]([O-])c1c([O-])[nH]c(-c2cccc3ccccc23)c1Nc1cccc(Cl)c1', 'Nc1[nH]nc2ncc(-c3ccc(-c4ccccc4)cc3)nc12', 'O=C([C@H]1CC[C@@H](C[NH3+])C1)[C@H](Cc1ccccc1)N1C(=O)c2ccccc2C1=O', 'Clc1ccc(Br)cc1-c1cn(-c2ccccc2F)nc1-c1ccccc1', 'Oc1ccccc1O[C@@H](N[C@@H](Cc1ccccc1)c1ccccn1)NC1(c2ccccc2)CC1', 'Clc1ccccc1Oc1cc(-c2ccc(O)cc2)nc2ccccc21', 'Clc1ccccc1[C@H](Cc1ccco1)N1CC[NH+](C2CCCC2)CC1', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1NCCN1C(=O)N[C@](CC2CC2)C1', 'O=C([C@H](c1c([N+](=O)[O-])nnc2sccc12)[NH+]1CCCC1)N1CCN(c2ncccn2)CC1', 'Clc1ccc(CC[C@H]2C[C@H]3CC[C@@H]2CN(C(=O)NCCC(=O)N4CCCC4)C3)cc1', 'ClCc1ccn(C[C@@](O)(c2ccccc2)N2CCOCC2)c1', 'Clc1ccc(C[NH+](Cc2ccccc2)Cc2ccccc2F)cc1', 'Clc1ccccc1C1(O)CC[NH+](C[C@@H]2C[C@@H]2c2ccccn2)CC1', 'Clc1ccccc1OC1([NH+]2CCCCC2)Cc2ccccc2C1', 'Clc1cccc2[nH]nc(NCc3ccc(Oc4ccccc4)cc3)c12', 'O=C(CNCc1c(F)cccc1F)N1CCN(Cc2ccc(F)cc2)CC1', 'Clc1ccccc1OC1CCN(C(=O)c2c[nH]c3ccccc23)CC1', 'Clc1ccc(-c2nonc2NCc2nc(-c3ccncc3)no2)cc1', 'O=C([C@H](C(=O)NC1CC1)N[C@@H]1CC(C)(C)c2ccccc21)N1CCN(c2ccc([N+](=O)[O-])cc2F)CC1', 'O=C(C[NH+]1CC[C@@H]2CCCCC[C@@H]21)Nc1ccc(I)cc2cccc12', 'O=C([C@H]1[C@H]2C[C@@H](c3ccccc3O1)N(Cc1nc3ccccc3n12)C(=O)[O-])N1CCN(c2ccccc2)CC1', 'Clc1ccc([C@@H](CCCCl)NC(=O)c2ccccc2)cc1', 'Clc1ccccc1OCC[NH+]1CCC[C@H]1c1ccc(F)cc1', 'Clc1ccccc1C1CC[NH+](Cc2cccc(Cl)c2Cl)CC1', 'O=C(CNC(=O)CCc1c[nH]c2ccccc12)c1ccccc1', 'O=C(c1cn[nH]c1N1CCC[C@@H]1c1ccccc1)N1CCC[C@@H]1c1nc2ccccc2[nH]1', 'O=C(Cc1nn(Cc2ccccc2)c2c1CCC2)N1CCCOCC1', 'O=C(C1CCN(c2ccccc2)CC1)c1ccc(-n2cccc2)cc1', 'Clc1ccccc1OCCn1cccc(Br)c1=O', 'Clc1ccccc1Cc1cnn(CC(=O)N2CC[C@H](Cc3ccc(Cl)cc3)C2)c1', 'O=C(CC[C@@H]1COc2ccccc2C1)NCCc1ccc(Cl)cc1', 'ClCc1csc(-c2cc(CN(Cc3ccccc3)c3ccccc3)oc2C2CC2)c1Cl', 'O=C(Cc1cnoc1)N1CCC(CC[NH+]2CCCCC2)CC1', 'O=C(/C(CO)(CC(=O)N1CCOCC1)c1ccoc1)c1ccccc1', 'OC[C@@]1(Oc2ccccc2-c2ccccc2)COc2ccccc21', 'Oc1cc(O[C@@H](C[NH+]2CCCC2)c2ccccc2)c(Cn2cccn2)c2ccccc12', 'Clc1ccccc1OCC1(C[NH2+]C2CC2)CCCCCCC1', 'Oc1ccccc1Oc1cccc[n+]1/N=C/c1nc2ccccc2o1', 'Clc1ccc(COc2nc3[nH]c4ccccc4n3c2NCCCCCCN2CCOCC2)cc1', 'O=[N+]([O-])c1c(Cl)nnc2c1CN(Cc1ccccc1Cl)[C@H]2NC(=O)c1ccccn1', 'Clc1ccccc1OCc1cccc(Oc2ccc(-n3cccn3)cc2)c1', 'O=C(CNc1nn(Cc2ccccc2)c2ccccc12)N1CCN(c2ncccc2F)CC1', 'Clc1ccccc1CSc1nc2c(cnn1-c1ccccc1)c1ccccc12', 'ClC[C@@H]1CN(Cc2cc(F)ccc2F)CC1', 'Clc1ccccc1Oc1cnc(NCc2cccs2)nc1CC[NH+]1CCC[C@@H](Cl)C1', 'Clc1ccc(C2(OC[C@@H]3CCN(C4CCCCC4)C3)CC2)cc1', 'O=[N+]([O-])c1c(Br)c(F)cc(OCC(=O)N2CCC[C@@H](Cc3ccccc3)C2)c1', 'Clc1nnn(C(F)F)c1NCC(=O)N1CCN(c2ccccc2F)CC1', 'Clc1cc(CC(=O)N2CCN(CCn3cncn3)CC2)ccc1N1CCCC1', 'O=C(C[NH+]1CCN(C(=O)c2c(F)cccc2F)CC1)N1CCCC1', 'Clc1ccccc1[C@H](Br)C1(C[NH+]2CCCCCC2)CC1', 'COC(=O)N1CC[C@@H]1CC(=O)Nc1ccccc1Oc1ccccc1F', 'O=C(CC[C@@H]1NC(=O)c2ccccc2N1CC1[NH+]=c2ccccc2=[NH+]C1=O)N[C@H]1CCCC[C@@H]1O', 'Clc1ccccc1C(C)CCC[NH+]1CCCCC1', 'Nc1cnn(CCC(=O)N2CC[C@@H](C(=O)[O-])[C@H]2C)c1[N+](=O)[O-]', 'CNC(=O)c1ccc(CNC(=O)CCc2c[nH]cc2C(C)(C)C)cc1', 'CC(C)OCCc1n[nH]c(-c2ccccc2)n1', 'N#C[C@H](NCc1ccc(N2CCOCC2)c(Cl)c1)C1(c2cc(Cl)ccc2Cl)CCCC1', 'O=C(c1cncc(N2C[C@@H](C(=O)NC3CC3)C2)n1)N1CCC[C@H](Cc2ccc[nH]2)C1', 'Clc1ccccc1CO[C@H]([NH2+]Cc1ccc(F)cc1)c1ccccc1F', 'Clc1ccc(Br)c(F)c1C[NH2+][C@@H]1CC[C@H](CCc2ccccc2)C1', 'Clc1cccc(CC[NH+]2CCN([C@@H]3CCN(CC4(O)CCOCC4)CCCC3)CC2)c1', 'O=C(CNc1nn2cccc2c2ccccc12)Nc1cccc(-c2nncn2C2CC2)c1', 'O=C(C1CC[NH+]([C@H]2CCO2)CC1)N1CCN(C(=O)Nc2ccccc2)CC1', 'Clc1cc(Br)cc(CN2CCO[C@@H](c3ccc(C(F)(F)F)cc3)C2)c1', 'Clc1cccc(F)c1C[NH+]1CCC[C@H]1C[C@@H]1CCC(=O)NC2=C1CCCC2', 'O=C(/C=C\\CC1CCCC1)NC[C@@](O)(c1cccs1)C1CC1', 'CC[S@](=O)[C@@H]1CCCC[C@@H]1NC(=O)N1CCC(OCc2ccc(Cl)cc2)CC1', 'O=C(c1c[nH]c2ccccc12)N1CCN(c2ccc[nH+]c2)CC1', 'O=C([C@H]1[C@H]2CCCCN2C1=O)N(c1ccc(-n2cnnc2)cc1)Cc1ccccc1', 'O=C(/C=C/O[C@@H]1CCS(=O)(=O)C1)Nc1ccccc1C(=O)N1C[C@@H](C)CC[C@H]1C(=O)NCC(F)(F)F', 'O=C(c1ccc(=O)n(Cc2ccccc2)c1)N1CCC(OC[C@H]2CCCO2)CC1', 'O=C([O-])[C@@H](c1ccccc1)[C@H]1CCCC[NH+]1Cc1ccc(Cl)cc1', 'O=C(c1cnn2nc(C3CC3)oc12)N1CCC([C@H]2CCOC2)CC1', 'Clc1ccc(Br)c([C@H]2[C@H]3CCCC(=O)N3C3CC[NH+](CCN4CCCCC4C2)C3)c1', 'O=C([C@H](CC1CC1)N1CCOc2ccccc2C1)OC[C@@H]1CCCCO1', 'O=C(CC[C@@H]1CN(C2CCCC2)CCN1Cc1ccccc1)Nc1ccccc1F', 'Clc1cccc2c1Oc1nc3c(O)c(O)ccc3nc1n2Cc1ccco1', 'Clc1ccccc1CN1CCC[C@@H](c2ccc[nH+]c2)C1', 'Clc1nnn(-c2ncn(-c3ccccc3)n2)c1-c1ccccc1', 'Clc1ccccc1-c1nn(-c2ccccc2)c(C(=O)N(Cc2ccccc2)C2CC2)c1C(F)(F)F', 'Clc1cccc([S@@]C(=S)Nc2ccc3ccc(=O)n(Cc4ccccc4)c3c2)c1', 'Oc1cc(O[C@@H](Cc2ccc(Br)s2)C(F)(F)F)c(OC[C@H]2CCCO2)cc1', 'Clc1cccc(Br)c1C[NH2+][C@H]1CCC[C@H](Cc2ccccc2)C1', 'O=[N+]([O-])c1ccc[nH+]c1-c1nc2ccc(OCC(=O)Nc3c(Cl)cccc3Br)nc2[nH]1', 'O=C(CCCCCn1cncn1)N1CCC[C@@H](COc2ccc(F)cc2)C1', 'O=C(C1=CCCN1c1cccc2cnccc12)Nc1ccc2c(c1)OCO2', 'Clc1ccccc1Oc1cc(-c2ccccc2)nc2ccc(F)cc12', 'O=C(COc1ccccc1)Nc1ccccc1N1CCCCC1=O', 'O=C(CC1(C2=Nc3ncnn23)[C@@H](CO)C1)N1CCCCC1', 'Clc1ccc(Br)c(Cl)c1C1CC[NH+](Cc2ccccc2)CC1', 'O[C@@H]([C@@H](N[C@@H](CO)c1ccccc1)c1ccccc1)c1ccccc1', 'Clc1ccc(CN[C@@H](C[C@@H]2CCCCO2)c2nccn2-c2ccccc2F)cc1', 'Clc1ccccc1O/Cn1[nH+]c(-c2ccccc2)nc1-c1ccccc1Cl', 'Oc1cc(-n2cnc(Cc3ccccc3)n2)ccc1Oc1ccc(Br)cc1', 'Clc1ccccc1N1N(CCc2ccccc2F)c2ccccc2C1', 'Oc1ccccc1Oc1ccccc1-c1nc(-c2ccccc2)nn1CC1CC1', 'Clc1ccccc1Nc1cnccc1C[NH2+][C@@H]1CCc2ccccc21', 'Clc1ccccc1OCc1ccc(CN2CCCn3cc(O)cc3C2)cc1', 'Oc1cc(CCn2c(=O)[nH]c3ccccc32)c(Cl)cc1OCc1ccccc1', 'Clc1ccc(C[NH+]2[C@H]3CCCC[C@H]3C[C@@H]2C2CCCC2)cc1', 'OCS[C@H]1CC[C@](NN=C2/C(=CCc3ccccc3)NN2Cc2ccccc2)N1', 'Clc1cccc2ncnc2c1/N=C\\c1ccc(COC2CCCC2)cc1', 'Oc1ccccc1C12CC(CC(COc3ccccc3)CS2)[C@@H]1S', 'OC[C@@]1([C@H]2CCCN(C(=O)c3ccco3)C2)C(=O)C[C@@H]1Oc1ccccc1', 'O=C(c1cc([N+](=O)[O-])c2ccccc2n1)N1CCCN(c2nnc(-c3ccc(F)cc3)o2)CC1', 'COC(CNC(=O)c1cc2ccccc2oc2ccccc12)c1ccccc1', 'O=C(COCCCCl)N1CCC[C@H]1c1ccccc1', 'Clc1ccc(Br)c(Cl)c1COCc1ccccc1Br', 'O=C(Cc1conn1)N1CCC[C@@H](c2nc3ccccc3n2CC(F)(F)F)C1', 'CNC(=O)[C@H]1CN(CC(=O)Nc2ccc(F)c(F)c2)CCO1', 'Clc1cccc(F)c1[C@@H](c1ccccc1Br)N1CCOCC1', 'Clc1ccccc1-c1cscc1-n1cc(-c2cccc(Cl)c2)cn1', 'Oc1ccc([C@H](OC(=O)Cc2ccccc2)N2CCOCC2)c2c1CCCC2', 'O=C([C@H](Cc1ccccc1)N1CCOCC1)[C@H]1CN2CCCC[C@@H]2CO1', 'O=C(C[NH+]1CCC[C@](O)(C[NH+]2CCCCC2)C1)N[C@@H]1CCOc2ccccc2C1', 'O=C(c1cn2ccsc2c1N[C@@H]1CCOC1)N1CCC[C@@H](c2nnc3n2CCCCC3)C1', 'O=C(CNc1nnccc1[N+](=O)[O-])Nc1ccc(CCC(=O)N2CCCCCC2)cc1', 'C[C@@H](c1nc2ccccc2[nH]1)C(=O)N1CCC(Cc2ccccc2)CC1', 'O=C(c1cn[nH]c1-c1ccccc1)N1CC(C(=O)NCCC23CC4CC(CC(C4)C2)C3)C1', 'Oc1ccccc1[C@H](O)[C@@H]([C@H]2CCOC2)N1CCN(c2ccccc2F)C1', 'O=C([C@H](Cl)c1[nH]ccc1C(F)(F)F)[C@@H]1CO[C@@H]1c1ccccc1', 'COCC[C@@H](C)C(=O)N[C@@H]1CC[NH+](Cc2ccccc2)C1', 'Clc1ccccc1C[NH+](C[C@]1(Cc2ccccc2)O[C@H]1CCO)C1CC1', 'Clc1ccccc1OC1CCC([NH2+]Cc2ccccc2CO)CC1', 'Clc1cccc([S@@]C(CC2CC2)C2CC2)c1', 'Oc1cccc(Nc2ccc(F)c3c2[nH]c2ccccc3c2=O)c1[O-]', 'O=C(CSCC1CCCCC1)N1CCC[C@@H](c2nc3ccccc3n2C2CC2)C1', 'N=C1[C@H](NCSc2ccc(F)cc2)C(=O)N(c2ccccc2)N1CCc1ccc(Cl)cc1', 'COc1nnn(CNC(=O)N2CCC[C@@H](C(N)=O)C2)c1N1CCCC1', 'Clc1ccc(Br)cc1-c1cn(-c2ccccc2)nc1-c1ccccc1', 'Clc1ccc([C@@H]([NH2+][C@@H](CO)c2ccccc2)C2CC2)nc1', 'Clc1ccccc1CNC1CC1', 'Oc1cc(Cn2nccc2-c2cccnc2)ccc1-c1cccnc1', 'Clc1ccc([C@@H](OC[C@@H](O)C2CC2)N2CCN(c3ccccc3)CC2)cc1', 'Oc1ccccc1/N=C(\\NCC(=O)N1CCN(c2ccc(Cl)cc2)CC1)C(=O)[O-]', 'O=C(CC1(C2=CC(=O)c3ccccc32)CC1)N1CCCC1', 'Oc1ccc(/C=N\\C2=C(C(=O)c3nncn3CCc3ccncc3)[NH+](C)C2)cc1', 'Clc1ccc([C@@H](CCc2nc([C@H]3CCCN3c3ncc[nH]3)no2)c2ccccc2)cc1', 'Clc1ccccc1[C@H](Cl)[C@](Cc1ccccc1)[NH+]1CCCC1', 'COc1nc(Nc2nc(C3CC3)n(-c3ccccc3C)n2)cc2c1OCO2', 'Clc1ccccc1Oc1cc(OC(c2ccccc2)c2ccccc2)cs1', 'Clc1ccc(Br)c(-c2n[nH]c([C@@H]3CCCN(/C=C/CCC[NH+]4CCCC4)C3)n2)c1', 'Oc1ccccc1[C@H]([NH2+]C[C@@H]1CC2(CCCC2)O1)c1ccccc1', 'O=C([O-])[C@@H]1CCCN(C(=O)Cc2c[nH]c3ccccc23)C1', 'CN[C@H](C1(CCC2CC2)CC1)c1ccc(C)cc1', 'Oc1cccc(SC2=[N+](c3ccccc3)Nc3cc(F)c(F)cc32)c1', 'COCCS[C@H](C)C(=O)N[C@H]1CCc2ccc(C)cc21', 'Clc1cccc([C@@H](N[C@H]2CCCc3ccc(F)cc32)C2CC2)c1', 'O=C(c1cccc2ccccc12)N1CCO[C@@H]2CCCC[C@@H]21', 'Clc1ccc([C@@H](OC)C(=O)N2CCC[C@@H](Cn3cc4ccccc4n3)C2)cc1', 'O=C(c1cccc(Cc2nnc(-c3ccccc3)o2)c1)N1CCOC(N2CCOCC2)C1', 'Clc1ccccc1[C@H]1[C@@H](C(=O)c2ccccc2)O[C@H]1c1ccccc1', 'Clc1ccccc1NCc1ccccc1N1C(=O)CCc2ccccc21', 'Clc1ccc(Br)c(OCCSc2ccc(Cl)cc2)c1', 'O=C([C@H](Cl)C1CC1)N1c2ccccc2C[C@@H]1CCc1ccc(C(F)(F)F)cc1', 'O=C(/N=C\\Nc1cccc(N2CCCC2)c1)c1ccc2c(n1)CCN(Cc1csc(Cl)c1)C2', 'Clc1ccc(CC2=NC(=O)/C3=NC4=CC(NC(=O)CS/C4=NC/C(=C/c4ccccn4)C3)C2)cc1OCCO', 'Oc1ccc([N+](=O)C2(CCO)Oc3ccccc32)c2ccccc12', 'O=[N+]([O-])c1ccn(Cn2c(=O)c3ccccc3n2Cc2ccccc2)c1', 'Clc1ccccc1COC(=O)c1ccc2c(c1)N(Cc1cc(F)cc(F)c1F)CC2', 'Oc1cc(-n2nnc(-c3ccc(Cl)cc3)n2)ccc1Oc1cccc(Cl)c1Cl', 'O=C(COc1cc(Cl)c2ncn(-c3ccccc3[N+](=O)[O-])c2c1)Nc1cc(C(F)(F)F)ccc1Cl', 'Clc1ccccc1-c1nn(Cc2ccnc(COC(F)(F)F)c2)nc1-c1ccc(Br)cc1', 'Clc1ccccc1-c1nc(C[C@H]2CCCO2)n[nH]1', 'O=C(c1c[nH]nc1CNc1ccc(-c2[nH+]c[nH]n2)cc1)N1CCOCC1', 'Clc1ccccc1-c1n[nH]cc1C[NH+]1CCC(C2CCN(C(=O)c3ccco3)CC2)C1', 'Clc1ccc(Br)c(Cl)c1C1CCN(Cc2cn3cc(-c4ccccc4)cc3nc2)CC1', 'Clc1ccc([C@@H](Br)CSCc2ccccc2)cc1', 'CCn1c(=S)[nH]c2ccccc21', 'Oc1ccc(O[C@H](C[C@@H]2CCCCO2)C(=O)c2ccccc2)c2c1C(=O)N=C(c1ccccc1)[NH+]2', 'COc1ncnc(N2CCN(c3ccccc3OCc3ccccc3)CC2)c1[N+](=O)[O-]', 'COc1nn(-c2ccccc2)c(C(=O)N[C@H]2C[C@]2(c2ccc(Br)cc2)c2ccco2)c1C', 'Clc1ccc(Br)c(O)c1O[C@@H]1CCCC[C@@H]1OCCc1ccccc1', 'Clc1ccccc1Oc1cnc(C[NH+]2CCC(c3noc4ccccc34)CC2)cn1', 'Clc1ccc(Br)cc1O[C@@H](c1ccccc1)N1CCCC1', 'Clc1ccc(Br)cc1N(C[C@H]1CCCO1)c1ccccc1', 'Clc1ccccc1N1N(Cc2ccccc2F)c2ccccc21', 'Oc1cc(-n2nncc2CN2CCO[C@@H](c3ccccc3)C2)ccc1N1CCCC1', 'Clc1ccccc1NCCN1CCC[NH+](C2CCCC2)CC1', 'O=[N+]([O-])c1c(-c2cc(O)ccc2[N-]S(=O)(=O)c2ccc(Br)cc2)sc2c1CCCCC2', 'Oc1ccccc1CCN1CCC([NH2+]Cc2ccc(CO)cc2)CC1', 'Clc1ccsc1-c1cn(CCC(=O)N2CCCCCCC2)c2ccccc12', 'O=C([C@H]1[C@H](c2ccccc2O)C=C(CO)C1=O)NC1CC[NH+](Cc2ccco2)CC1', 'C=C(Br)CN(C(=O)c1cccn1CC)[C@@]1(C)CCCO1', 'C[C@@H](N[C@@H]1CC(C)(C)COC1)C(=O)N[C@@H]1CCN(c2c(F)cccc2F)C1', 'O=[N+]([O-])c1c(OCc2[nH]c3ccccc3n2)nc2ccccc2c1N1CCOCC1', 'Clc1ccc(C[NH+](C[C@@H]2Cc3ccccc3O2)c2nccn2Cc2ccccc2)cc1', 'N[C@@H](Nc1nc2c(s1)C[NH+](C1CC1)CN(Cc1cccs1)C2)C1CC1', 'Clc1cccc(Br)c1C[NH+]1CCCC[C@@H]1CCl', 'O=C(c1cccn2cc(Cl)cc12)N1CCC(C(=O)OCc2ccccc2)CC1', 'O=C(c1c[nH]nc1NC[C@H](c1ccco1)N1CCc2ccccc21)N1CCOCC1', 'Clc1ccc(Br)c(/N=c2\\[nH]c3cnccc3n2-c2ccccc2)c1', 'NS(=O)(=O)N1CCC2(CC1)Oc1ccc(F)cc1C(c1ccccc1)(CCC[NH+]1CCOCC1)CO2', 'CC1CCN(C(=O)c2cc(C(=O)N(Cc3ccccc3)c3ccccc3)ccc2F)CC1', 'O=C(CN(C[C@@H]1CCCO1)c1ccc(Cl)cc1)N1CCOCC1', 'Oc1cccc(Br)c1OC1(c2ccccc2)CCCC1', 'O=C(c1c[nH]c([O-])n1)N1CC(c2nc(-c3ccco3)nn2CC(F)F)c2ccccc21', 'Clc1ccccc1C[NH+](C1CC1)[C@@H]1CCC[C@H](C(F)(F)F)C1', 'Clc1ccccc1OCCS[C@H]1[C@@H]2CCC[C@@H]2C1', 'N=Cc1cnn(-c2ccc(F)cc2F)c1NC(=O)C(=O)NC(=O)c1ccc2ccccc2c1', 'Clc1cc(Br)cn1CCNC(=O)C1CCN(S(=O)(=O)c2c(F)c(F)c(F)c(F)c2F)CC1', 'Clc1ccc(-c2ccccc2N2c3ccc(F)cc3C(=O)N2C2CCCCC2)cc1', 'O=C(C[NH+]1CCC[C@H]1COc1ccc2n[nH]c(=O)cc2c1)N1CCCCC1', 'O=C(c1cn(C[C@@H]2CCCO2)cc1O)[NH+]1CCC(CN2CCOCC2)CC1', 'Oc1ccc([C@H](NNC2CC[NH+](Cc3c[nH]c4ncccc34)C2)c2ccccc2)cc1', 'Clc1ccccc1C(C1CC1)[C@H]1CCC[C@@](O)(Cn2cccn2)C1', 'Clc1cc(/C=C/C[NH+](Cc2ccco2)Cc2ccco2)n(C2CC2)c1', 'O=C([C@H](c1c(F)cccc1F)N1CCN(c2ccccc2)CC1)N1CCCC1', 'Oc1ccccc1[C@@H](Br)[C@@H](CO)Cc1ccccc1', 'Clc1cc(Cl)ccc1-n1cccc1C1CC[NH+](Cc2ccccc2)CC1', 'O=C(COc1cc(Br)c(O)cc1)N1CCN(S(=O)(=O)c2ccc(Br)cc2)CC1', 'O=C(C1=NN(c2cccc(O)c2)C(=O)N=C1[O-])N1CCO[C@@H](c2ccccc2)C1', 'O=C(c1c[nH]nc1Nc1ccc(C(=O)NC2CC2)cc1)c1ccccc1', 'O=C(c1ccc(S(=O)(=O)N2CCOCC2)cc1)N1CCN(c2nc3ccc4ccccc4c3c(COc3ccccc3)nc2=O)CC1', 'Clc1cccc(Br)c1NC1CC[NH+](CCN2CCCC2)CC1', 'Clc1ccccc1NCc1ccccc1CN1CC[NH+](CC2(CCO)CC2)CC1', 'O=[N+]([O-])c1c([O-])[nH]c(C[NH+]2CCC(n3cccn3)CC2)c(-c2ccccc2)c1=O', 'Clc1ccc(Br)c(Cl)c1[C@@H](CO)NC1(N2CCOCC2)CCCC1', 'Clc1ccc(Br)cc1S[C@H]1C[C@H](C2CC2)C1', 'Clc1ccc(-c2nn(-c3ccccc3)cc2CN2CCN(CCc3cccc(F)c3)CC2)cc1', 'O=C(/C=C1CC(=O)c2ccccc2[C@@H]1c1c[nH]c2ccccc12)c1cccc(Cl)c1', 'Oc1ccccc1-c1c(-c2ccc3c(c2)OCCO3)n[nH]c1N1CCOC[C@@H]1C1CC1', 'O=C([C@H](Nc1ccc(F)cc1)C(=O)N1CCN(c2ccccc2)CC1)N1CCCC1', 'Clc1ccc(CCN(Cc2ccco2)C[C@@H]2C=CCCC2)cc1', 'Clc1ccccc1C[NH+](C1CC1)[C@H]1CCCc2ccccc21', 'Clc1ccccc1C(=O)O[C@@H]1CN(C2CC2)CCN1Cc1ccccc1', 'O=C(Cc1coc2nccc(NC(=O)Cc3ccccc3)c12)c1ccc(Cl)cc1', 'O=C([C@H](CNc1ncccn1)[NH+]1CCCC1)NN[C@@H]1CCS(=O)(=O)C1', 'COC[C@@H]([NH2+]CC1CCN(Cc2ccccc2F)CC1)C(=O)OC', 'Clc1ccccc1Cc1c(Cl)ncnc1N[C@@H](C(=O)c1ccccc1)[C@H]1CCCO1', 'Clc1ccc(-c2nn([C@@H](c3ccccc3)[NH+]3CCC(OC(F)F)C3)cc2F)cn1', 'O=[N+]([O-])c1cc([C@H]2OCCN2CCCN2CCOCC2)c2cn[nH]c2c1', 'N=Cc1cn([C@@H]2CCC[C@]2([NH+]2CCCC2)c2ccccc2)nc1C(F)F', 'Clc1ccccc1[C@H]1C[C@H](CCCO)CN1C(=O)c1cc2c([nH]c1=O)CCC2', 'O=C(Cc1nn(C[C@H]2CCCO2)c(=O)c2ccccc12)NCc1ccccc1', 'Oc1cc(-n2nnnc2-c2ccccc2F)cc(-c2ccsc2)n1', 'O=C(c1ccsc1C(F)F)N(CC(=O)N1CCCCCC1)c1ccccc1', 'Nc1[nH]nc(CN[C@@H]2CC[C@@H]([NH+](Cc3ccccc3)C(=O)NCCCC(=O)[O-])c2)c1Cl', 'O=C(/C(CO)(CC1CC1)c1ccc(Br)cc1)c1ccccc1', 'O=[N+]([O-])c1c(Cl)nn(C[C@H]2CCCO2)c1N1CCN(Cc2ccccc2)CC1', 'O=C(CC[NH+]1CCC[C@H](c2nc(C3CC3)no2)C1)Nc1ccc(Cl)cc1', 'Clc1nncn1-n1cnc(COCCN2CCOCC2)c1C1CC1', 'O=[N+]([O-])c1cc(C2(c3ccccc3Cl)CC2)cn1Cc1ccccc1', 'O=C(CCCNC(=O)C1(c2ccc(Br)cc2)CC1)Nc1ccccc1', 'O=C(c1ccsc1)c1cn(-c2ccc(F)cc2)nc1-c1ccccc1[N+](=O)[O-]', 'Clc1ccccc1Nc1cc(-c2ccccc2)oc1N1CCN(c2ncccn2)CC1', 'Clc1ccccc1NCC(=O)N1CCCCC1', 'Clc1cc(Br)c(C2=NC[C@@H]2CN2CCO[C@H](CCO)C2)cc1Cl', 'O=C(c1c[nH]c2ccccc12)N1CCC(OC[C@H]2CCCCO2)CC1', 'O=[N+]([O-])c1ccc[nH+]c1C[NH2+][C@H]1C(CO)=C(C)CC(=O)N1c1ccc(Br)s1', 'O=[N+]([O-])c1c(Cl)n2nnc(S[C@@H]3C=CC[C@@H]3CN3CCOCC3)c12', 'O=C(c1ccc[nH]c1=O)N(CC1=CCCCC1)C1CCCCC1', 'C=C1c2ccc(OC)c(C(=O)NCc3ccccc3)c2[C@H]1c1ccc(SC)cc1', 'Oc1cc(Br)c2c(c1)nc(C[C@@H](CO)O)n2CC1CCN(c2ccccc2)CC1', 'Clc1ccc(-c2nnnn2-c2cc(OCc3cccnc3)ccc2[N+](=O)[O-])cc1', 'Clc1ccccc1OCc1cc(-c2ccccc2O)nc2ccccc12', 'O=C(c1cnc(Cc2ccccc2)s1)N1CCC2(CC1)Nc1ccccc1C2=C1CCOCC1', 'O=C([O-])C[C@@H](N[C@@H]1CCc2c(Br)cccc21)C(=O)[O-]', 'Clc1ccc(Br)c(S(=O)(=O)N2CCN(c3ccc(Cl)c(C(=O)[O-])c3)CC2)c1Cl', 'C[C@@H](c1noc(-c2ccc(O)cc2)n1)N1CC[NH+](C[C@@H]2CCCO2)CC1', 'ClC[C@H](CN1CCN(c2nc3ccccc3nc2C(F)(F)F)CC1)N1CC[NH+](CCCO)CC1', 'Clc1cccc([S@@]C(C2CC2)C2CC2)c1', 'Clc1ccc(-c2ccccc2CCNC(=O)COc2ccccc2Cl)cc1', 'Clc1ccccc1NS[C@H]1[NH+](C2CC2)CCC[NH+](C2CC2)[C@@H]1C', 'O=C(/N=C/CC(=O)Cc1ccccc1)[C@@H]1CN(Cc2ccccc2)CCCO1', 'Clc1cccc(Br)c1C(F)(F)C[NH+]1CCCC1', 'O=C(/C([O-])(F)F)c1ccc(OC(F)(F)F)cc1OCO[C@@H]1CCc2cc(OCc3ccccc3)ccc21', 'CNc1nc(C(=O)N2CCC[C@H](CNc3ccc(OC)cc3)C2)c2ccccc2[nH+]1', 'O=[N+]([O-])c1cc(SC2CCN(c3ccccc3)Cc3ccccc32)c2[nH]ccc2c1', 'Clc1ccccc1O/N=C\\c1ccc(COc2ccccc2)cc1', 'O=C([O-])[C@H](Cc1ccc(F)cc1)NC=C1C[NH2+]CCc2ccccc21', 'O=C(CN(CCc1ccccc1)C1CC1)Nc1ccccc1Cl', 'O=C(c1cnc(OCCc2ccccc2)cc1)N1CCC[C@@H](OCc2ccccc2F)C1', 'O=C(/C=C/c1c(N[C@@H]2CCSc3ccccc32)[nH]c2ccccc12)NCc1ccc(C(=O)OCCO)o1', 'O=C(COCc1nnc2n1CCN(Cc1ccccc1Cl)CC2)N[C@@H](c1ccc(Cl)cc1)C(F)(F)F', 'Clc1ccccc1-c1cscc1-n1c([C@H]2CCCO2)cc2ccccc21', 'CCn1nnnc1[C@@H]1CCCCN1C(=O)CN(C)C(=O)c1cccnc1', 'CCS[C@@H]1CCCCN(Cc2ccccc2N2CCCC2)C1', 'O=C(c1c[nH]nc1-c1ccccc1)N1CC[C@@H](c2ccccc2)[C@H]1Cc1cccs1', 'Oc1ccc(/C=C/N=[N+]2CCC[C@H]2SC2CCCCC2)cc1', 'COc1nc2snc(/C=C(Cl)C3=NOCCO3)c(=O)c2cc1CN1C[C@H](C)O[C@H](C)C1', 'Clc1cccc(-c2cnc(-c3ccc4ccccc4c3)o2)c1', 'Clc1ccccc1CO[C@H](c1ccccc1)c1ccccc1', 'O=[N+]([O-])c1cc(N2CCC(Oc3cc(F)cc(Cl)c3)CC2)ccc1OC[C@@H]1CCCO1', 'CNC(=O)[C@@H]1CCCCN1c1ccc(NC(=O)[C@H](C(C)C)[NH+]2CCCCC2)cc1', 'Clc1ccc(C(C)C2CCN(C(=O)[C@H]3COc4ccccc4O3)CC2)cc1', 'Clc1ccccc1Oc1cnc2nnc(-c3ccncc3)n2c1', 'Oc1cc(Cn2c[nH+]c(-c3ccccc3)c2NCc2ccco2)ccc1F', 'Clc1ccccc1[C@H](CNC(=O)Nc1ccc(Br)cc1)N1CCOCC1', 'O=C(CNCc1noc(COc2ccccc2)n1)N1CCOCC1', 'OC[C@H]1C(=O)C[C@H](C(=O)N(Cc2ccc(Cl)cc2)c2ccccc2)C1', 'O=C(C1CC[NH+]([C@H](c2ccccc2)[C@H]2CCCO2)CC1)N1CCOCC1', 'Clc1ccccc1OC1(C(=O)[O-])CCN(Cc2ccc(Cl)cc2)CC1', 'O=C([C@H](c1ccccc1Br)N1CCOCC1)N1CCC(c2ccncc2)(C(=O)[O-])CC1', 'O=C(/C=C1CCCCC1)Nc1ccccc1N(Cc1ccccc1)C1CCCCC1', 'Clc1ccc(-c2nnnn2[C@@H]([NH2+]CCCN2CCCC2)C2CC2)cc1', 'O=C(Cc1c(-c2ccccc2)nn([O-])c1Cl)C(=O)N1CCN(c2ccccc2)CC1', 'Oc1ccc(F)c(-c2nc(C(=O)N3CCCN(C4CC4)CC3)cnc2-n2cccc2)c1', 'Clc1ccc(CCN2CCN(c3cccc[nH+]3)CC2)cc1', 'Clc1ccc(CNCCCn2ccc[nH+]2)cc1', 'Clc1ccc(C(CCBr)OCc2ccccc2)cc1', 'Clc1ccccc1N1N(CC(=O)NCc2ccc(F)cc2)Nc2ccccc21', 'Oc1ccccc1[C@@H]([O-])CCCNC(=O)[C@H]1COc2ccccc2O1', 'O=C([C@H](Nc1nc(-c2ccccc2)n[nH]c1=O)Nc1ccccc1)N1CCCCC1', 'O=C(c1cc(F)cc2cc[nH]c12)N1CCO[C@@H](c2ccccc2)C1', 'Clc1cc(Br)c(Nc2ncnc3ccccc23)cc1/C=N/c1ccccc1', 'Clc1ccccc1[C@H]1[C@@H]([C@H]2CCS(=O)(=O)C2)NC2=C1C1CC[NH+](CCc3ccc(F)cc3)C=C12', 'O=[N+]([O-])c1c(F)cc(NCCCSC2CC2)cc1', 'O=C([C@H](c1c([N+](=O)[O-])cccc1[N+](=O)[O-])N1CCSCC1)N1CCCC[C@H]1C1CC1', 'Clc1ccc([C@@H](NCCC2CCOCC2)c2ccccc2)cc1', 'Oc1ccc([C@H](CNC(=O)c2cnc([C@H]3CCCCO3)n2C2CC2)[NH+]2CCCC2)s1', 'Oc1cc(Cn2nc(I)c3ccccc3c2=O)ccc1N1CCC2(CC1)OCCO2', 'O=C(c1c[nH]c([N+](=O)[O-])c1)N1CCO[C@@H](c2ccccc2F)C1', 'O=C(Cc1cc2c(cc(Cl)cc1[N-]S(=O)(=O)c1ccccc1)n2Cc1ccccc1)NCc1ccccc1', 'O[C@@H](C[NH+]1CCC[C@H]1c1ccccc1)C1CC1', 'CC1(C)[C@H]([NH2+]CNc2ccccc2)C(C)C1', 'O=C(Cc1coc(-c2cccs2)n1)N1CCN(C[C@H]2CCCO2)CC1', 'Clc1ccccc1C(C)(C)NCc1nc(C2CC2)n2ccccc12', 'NS(=O)(=O)N[C@@H](CCCn1cc[nH+]c1)c1ccc(F)cc1', 'Clc1ccccc1OCCOc1ccccc1Br', 'O=C(COCc1n[nH]c(=O)c2ccccc12)N1CCCCCC1', 'Oc1cc(COC[C@@H]2CC[NH+](Cc3ccccc3)C2)ccc1OCc1ccccc1', 'O=C(CC1([C@@H](O)c2ccccc2)CC1)N1CCC2(CC1)OCCO2', 'O=C(C1CCC(Nc2ccccc2Cl)CC1)c1ccc(Cl)cc1', 'O=C(CC[C@@H](O)c1ccccc1)N[C@H]1CCCOc2ccccc21', 'O=C(CSCN1C(=O)[C@H]2CCN(Cc3ccccc3)C2=C1CN1CCOCC1)c1ccccc1', 'O=C(CN(c1cncn(-c2ccccc2F)c1=O)NN1C(=O)COC1=O)c1ccc2c(c1)CCCC2', 'Clc1ccccc1OCCOCCN1C(=O)NC2(CCCCC2)C1=O', 'Oc1ccc(/C=C(\\Cl)/C=C(Nc2ccc(Cl)cc2)N2CCOCC2)cc1', 'O=C([C@H](SC1CCCC1)c1ccccc1)Nc1ccc(Oc2ccccc2)cc1', 'Clc1ccccc1OCc1ccc(Oc2ncnc(Nc3cccc(F)c3)c2)cc1', 'CN=C1C(=O)N=C(N2CC(=O)N(Cc3cccnc3)C2=O)N=C(N)N1c1ccccc1', 'O=C(c1ccc(OCc2ccccc2)cc1)N1CCc2ccccc21', 'O=C(Cc1ccc(Cl)cc1)NC[C@H]1CCCN(c2ncnc3ccccc23)C1', 'O=[N+]([O-])c1ccc2cccnc2c1C[NH+]1CCC2=C(N2c2ccccc2)CC1', 'O=C(c1ccsc1)Nc1nnc(C(=O)Nc2ccc(-c3ccccc3)cc2)s1', 'COc1nc(N(C2CC2)[C@@H](C)C2CC2)ccc1-c1ccco1', 'COc1nnnc(N(Cc2ccccc2)CC(=O)c2ccc(Cl)cc2)c1O', 'Oc1ccc(F)cc1-n1cnc2ccc(Nc3ccccc3)cc2c1=O', 'O=C(c1c[nH]nc1C[C@H]1CN2CCCC[C@@H]2CO1)NCc1nc2ccccc2c(=O)[nH]1', 'Clc1cccc2[nH]c(=O)n([C@@H](CN3CCO[C@H](O)CCCO)C3)c12', 'O=C(CNc1cn(C2CCCCC2)c[nH+]c1=O)Nc1cccc(Cl)c1Cl', 'O=C(c1c[nH]nc1C[NH+](CC1CC1)C1CC1)N[C@H]1CCN(c2ccc([N+](=O)[O-])cc2)C1=O', 'O=C(/N=C/CCOCc1ccccc1)NC1CC[NH+](Cc2ccccc2)CC1', 'Nc1noc(C[NH+]2CCC[C@H](c3nnc(Nc4cccnc4)n3N)CN2)c1Cl', 'O=[N+]([O-])c1c([O-])n(-c2ccccc2)c2ccccc12', 'Oc1cccc(N2C[C@@H](C(=O)N3CCc4sccc4[C@@H]3c3ccccc3)CC2=O)c1', 'COc1nn[nH]c1CNC(=O)NCc1ccc[nH+]c1N(C)C', 'O=C(c1cccn2cnnc12)N(Cc1ccccc1)CC1CCOCC1', 'Oc1ccc(/C=N\\c2ccccc2)c[nH+]1', 'Oc1ccc([C@H](O)c2ccco2)c(F)c1', 'O=C(/N=C\\c1cc(Cl)ccc1N1CCOCC1)[C@H]1CC(=O)N(c2ccc(OC(F)(F)F)cc2)C1', 'Clc1ccccc1-c1nn(Cc2ccccc2)nc1N1CCC(c2ccccc2)CC1', 'O=C(c1ccsc1)c1nnn2c1Nc1ccc(Cl)cc1C(=O)N2CCc1ccccc1', 'Clc1nnnn1c1c(NC2CCN(c3ccccc3)CC2)ncn1-c1ccccc1', 'O=C([O-])/C(=C/c1ccccc1)NC[C@@H]1CCCO1', 'ClC(=S)c1ccc(N2CCC(COc3ccccc3)CC2)cc1', 'Clc1ccc(-c2cc[nH]n2)nc1SC[C@@H]1CN(C2CC2)CCO1', 'Oc1ccc(/C=C(\\Nc2ccccc2)C(=O)Nc2ccccc2)cc1', 'Clc1ccc([C@@H]([O-])N[C@@H](CC(F)(F)F)c2ccccc2)cc1', 'C[C@@H]([NH2+]CC1([NH+]2CCOCC2)CCCCC1)c1nc2ccc(-c3ccccc3)cc2[nH]1', 'O=C(/N=c1\\occcc1OCCCCc1ccccc1Cl)c1cnn(-c2ccccc2)c1', 'O=[N+]([O-])c1ccc2c(c1)[C@H]1C=CC[C@@H]1[C@@H](c1cccc(OCc3ccccn3)c1)N2', 'O=[N+]([O-])c1ccn(CCC[NH+]2CCCC[C@@H]2C2CCCCC2)c1', 'Clc1cccc(Br)c1COCc1ccc(Cl)cc1', 'Oc1ccccc1C[NH+]1CCC[C@@H]1CN1CCCC1=O', 'Clc1ccccc1CS(=O)(=O)N1CCC([NH2+]C2CCN(C(=O)CCCc3ccccc3Cl)CC2)C1', 'O=[N+]([O-])c1c([O-])c2c(c1-c1ccccc1)c([O-])c(=O)n2CCc1ccccc1', 'O=[N+]([O-])c1c(F)cccc1NCC(=O)N1CCc2ccccc2C1', 'Clc1cccc2c1[C@H](c1ccccc1)N2C(=O)Cc1ccccc1C(F)(F)F', 'Clc1ccc(Br)cc1-n1ncc2ccccc21', 'Clc1ccc(Br)cc1-n1cnc2ccccc21', 'Clc1ccc(Br)cc1-n1cnc2ccccc21', 'Clc1cccc([S@@]CC[NH+]2CCCC2)c1', 'O=C(COCc1ccccc1)N1CCC(c2ccccc2)CC1', 'OC1(SCCC(=O)Nc2ccc(Br)cc2Cl)CCCCC1', 'CC[S@](=O)Cc1cc(NC(=O)[C@@H]2CCCN2C(=O)N2CCCC2)cc(F)c1', 'COc1nnn(CC(=O)N2CCC[C@@H](O)C2)c1Cl', 'Clc1ccc([C@@H]([NH2+]C2CCN(C[C@@H]3CCOC3)CC2)C2CC2)cc1', 'ClC([O-])=C1CCC(n2c(=O)[nH]c3ccccc32)CC1', 'Clc1ccc(-c2nn(-c3ccc(-c4cccs4)cc3)nn2)cc1', 'O=C(C1CCCCN1c1ccc2c(c1)COC2)NN1CCC([NH+]2CCCC2)CC1', 'O=C([C@H](c1c(F)cccc1Cl)N(Cc1c[nH]c2ccccc12)C(F)(F)F)N1CCCC1', 'O=[N+]([O-])c1c(O)nc2ncnc(NC3CCC3)c2c1C[NH+]1CCCCC1', 'Oc1ccccc1[C@@H]1CN(c2ccccc2[N+](=O)[O-])CCN1CCO', 'Clc1ccccc1Oc1cnc2nn(-c3ccccc3)nc2c1', 'Clc1ccccc1Oc1cnc2snc(-c3ccc(Br)cc3)c2c1', 'Clc1ccccc1Oc1cnc2sc(-c3ccc(Br)cc3)nc2c1', 'O=C(CCNC(=O)Cn1c(N(C)Cc2ccccc2)nn(-c2ccccc2)c1=O)N1CCOCC1', 'O=C(Cc1conc1-c1cnn(C(C)(C)C)c1)N1CCC(NC(=O)c2cccnc2)CC1', 'Clc1cccc([C@@H](CCc2c[nH]c3ccccc23)N=C2CCOCC2)c1', 'Clc1ccc(COC[C@@H]2CCCN2C(=O)c2ccccc2Cl)cc1', 'CC(C)C[C@@H](C[NH3+])C(=O)Nc1ccncc1', 'Clc1ccccc1CO[C@H]([NH3+])Cn1ccccc1=O', 'Clc1ccccc1N1N(CCNC(=O)c2cccc3ccccc23)CC1', 'O=C([C@H](c1c(F)cccc1Cl)N1CCOCC1)N1CCCCC1', 'COc1ncnc(N[C@H]2[C@@H]3CC[C@H](C2)[C@H]2C(=O)N[C@H]2C3C(=O)C(C)(C)C)c1O', 'Clc1ccc(-c2nn([C@@H]2CC23CC[NH+](Cc4ccccc4F)CC3)C2)cc1F', 'Oc1cccc(N2CCCN(Cc3ccnc4c3cnn4-c3ccccc3)CC2)c1', 'O=C(COCCNc1ccccc1N1CCOCC1)NCc1ccccc1', 'O=C(c1c[nH]nc1-c1ccccc1F)N1CCO[C@@H](c2ccccc2)C1', 'COc1nn(C)cc1CN(C)C(=O)N[C@@H]1CCN(c2cnn(C)c2)C1', 'Oc1cccc(-c2cc(CNc3ccc(Cl)cc3)nn2-c2ccccc2)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(F)c1C(=O)NCCCc1ccccc1Cl', 'Clc1ccccc1NC(=O)c1ccc2c(c1)N(Cc1ccccc1C(F)(F)F)CCO2', 'Clc1ccc(-c2cc([C@H](Nc3cncc(F)c3)Nc3ccc(F)cc3F)[nH]c2)cc1', 'Clc1cc(/C=C(\\Nc2ccccc2)c2ccccc2)ccc1Cl', 'Clc1ccccc1OC1(N2CCOC[C@@H]2c2ccccc2)CC1', 'Clc1ccccc1C1CCN(CCSc2ccccc2)CC1', 'O=C([C@H](CSC(C)=O)c1ccc(F)cc1)N1CCc2ccc(F)cc2C1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(NCCC[NH+]2CCCCC2)c1Cl', 'Clc1ccccc1CNC1(c2ccccc2)CC1', 'Clc1ccccc1Oc1cnc2cc([C@H]3CCCN(C(=O)c4ccccc4)C3)[nH]c2c1', 'Clc1cccc2[nH]nc(N[C@H]3CCCC[C@@H]3[NH2+]C[C@@H](O)c3ccccn3)c12', 'Clc1ccccc1OC1(C[C@@H]2CCCCCC2)CC1', 'Clc1ccccc1COC1CCCCC1', 'Clc1ccc(C(=O)N[C@@H]2CCC(Cn3cc[nH+]c3)CCOCC2)cc1', 'O=C(Cc1c([O-])c2ccccc2oc1=O)Nc1ccc(Oc2ccccc2)cc1', 'O=C([C@H](CC1CC1)N(Cc1ccc(Cl)cc1)C1CC1)NCC(=O)Nc1ccccc1', 'O=C([C@H](NSCc1cc(F)c(F)c(F)c1)N1CCOCC1)NCC1(CO)CC1', 'O=C(c1c[nH]c2ccccc12)N(CCO)c1ccccc1', 'O=C(COCC(=O)N1CCOCCC1)Nc1ccc2[nH]c(=O)[nH]c2c1', 'O=C(c1c[nH]c2nc[nH]c(=O)c12)N(Cc1ccccc1)c1ccccn1', 'Clc1ccc(-c2ccccc2NC2CCCCC2)cc1', 'COc1nn(CCC(=O)N[C@@H](C)C[C@H]2CCCCO2)c2ccccc12', 'O=C([C@H](CNc1ccc(C(F)(F)F)cc1)c1cccs1)NNC(=O)c1ccc2c(c1)OCO2', 'Clc1ccc([C@@H]([NH2+]Cc2n[nH]c3ccccc23)C2CCOCC2)cc1', 'Clc1ccccc1COCC[NH+]1C[C@@H](C)[C@H](Oc2ccccc2C(F)(F)F)C1', 'O=[N+]([O-])c1c(OCCO[C@@H]2CCO[C@@H]2c2ccccc2)c2ccccc2o1', 'C[C@@H](c1ncc(C(=O)[O-])n1C1CC1)c1occc1COC(F)(F)F', 'Clc1ccc(-c2ccccc2[N-][C@@H]2CN[C@@H](c3ccc(Cl)cc3)C2)cn1', 'Cc1c([C@@H](C)NC(=O)c2cc(C)n(-c3ccc(Cl)cc3)c2C)c(=O)n(-c2ccccc2)[nH]1', 'N[C@@H](CC[NH+]1CCN(c2ccccc2)CC1)c1ccc(Cl)cc1', 'OC[C@H]([NH2+]Cc1ccccc1)C1CC1', 'C[C@@H](N[C@@H](CO)C(C)C)c1ccccc1', 'Oc1ccccc1[C@H]1C2C[C@@H]3CCCCC[C@H]3C[C@@H]21', 'ClC[C@@H]1OCC[C@H](Sc2cccc(Cl)c2)C1', 'Clc1ccccc1OCCC[NH+]1CCCCC1', 'Clc1cc(N2CCc3c(c(OCc4ccccc4)nn3Cc3cccc(F)c3)C2)ccc1[N+](=O)[O-]', 'Clc1ccc(-c2nnnn2[C@@H](Cc2ccccc2)c2noc(C3CC3)n2)cc1', 'Clc1cccc2ccc(COCCNC(=O)CC[C@@H]3CCC[NH+](CC4CC4)C3)nc12', 'CCNc1cccc(NC(=O)NC2CN(C(=O)CCc3ccc(C)cc3)C2)c1', 'Clc1cccc2cccnc1n2CCC(=O)N[C@H](c1ccccc1)C1CCCC1', 'COc1nn[nH]c1CCCNC(=O)c1ccnc(Br)c1', 'O[C@@H]1C[C@H](c2cccs2)C2(CCCCC2)N1[C@@H](Cn1cccn1)N1CCOCC1', 'O=C(/C=C/c1c(Cl)cccc1Cl)NC[C@@H]1CN2CCCC[C@H]2CO1', 'Oc1ccccc1-n1cccc1C[NH2+]Cc1cnn(-c2ccc(F)cc2)c1', 'Clc1ccccc1O/C=C[C@@H](O)COCc1ccccc1', 'Oc1ccccc1CN1CC[NH+]([C@H]2CCOc3ccccc32)CC1', 'Clc1ccc([C@@H]([NH2+][C@@H](CO)c2ccccc2)C2CC2)cc1', 'Oc1ccc(O[C@H](Cc2ccc[nH]c2=O)[C@@H]2CC[NH2+]C2)cc1', 'O=C(Cc1coc(-c2ccc(Cl)cc2)n1)N1CCCC[C@@H]1c1ccccc1', 'O=C(CSC(=O)C1CCCC1)NC[C@@H](c1ccc(Br)cc1)N(CC1CC1)S(=O)(=O)c1ccccc1', 'O=C([C@H](c1ccccc1)N(CC(F)(F)F)C1CCCCC1)C1CCCC1', 'Clc1ccccc1OCC1CC[NH+]([C@@H]2Oc3ccccc3O2)CC1', 'Clc1ccccc1Nc1cn([C@H]2CCOC3(CCOCC3)C2)[nH]c1=O', 'O=C(CN(Cc1ccccc1)c1ccccc1)NNc1cccc2ncccc12', 'O=C(Cc1nc2ncccn2c1C[NH2+]C1CC1)NC1CCCCC1', 'Clc1ccccc1C(=O)N[C@H](C(=O)c1ccco1)[C@H]1CCCO1', 'O=C(COCc1ccccc1)N/N=C/c1ccc(OCc2ccccc2)cc1', 'Clc1ccccc1OCCC1CCCCC1', 'O=C(c1c[nH]nc1N(Cc1ccccc1)Cc1ccc(Cl)cc1)C1CC1', 'O=C(c1cn[nH]c1)N1CCN(c2ccccc2CNc2nc3ccc(Cl)cc3s2)CC1', 'O=[N+]([O-])c1cc(N2CCN(Cc3ccccc3N3CCOCC3)CC2)nc(-c2ccco2)n1', 'Clc1ccccc1CC[NH+]1CCCN(Cc2cccc(F)c2)CC1', 'Oc1ccc([C@H](/C=C/c2ccc(Br)cc2)c2ccccc2)cc1', 'O=C(COc1cc(Cl)ccc1N1CCC(=O)[C@@H]1c1ccccn1)N1CCOCC1', 'Clc1nnnc(NC2CCC(Oc3ccccc3)CC2)c1C1CC1', 'Clc1ccccc1Cn1nncc1-c1ccccc1Cn1cccn1', 'Clc1ccccc1NCc1cc2ccccc2n1C[C@@H]1COCCO1', 'O=[N+]([O-])c1cc(C[NH+]2CCC[C@@H](CN3CCOCC3)N(c3ccccc3)C2)ccn1', 'O=C(/C=C/c1c(N)n(CCCO)c1=O)c1ccccc1', 'Clc1ccccc1N1N(CCc2cccc(Cl)c2)C=C1COc1ccccc1F', 'Clc1ccc(-c2nnnn2CCC(=O)N2CCCC2)cc1', 'Cc1cn(-c2ccccc2)c(C(=O)NC[C@H](c2ccccc2)[NH+]2CCCCC2)c1', 'CC[S@](=O)[C@H]1CCCC[C@@H]1NC(=O)Nc1ccn(Cc2ccc(Cl)cc2)n1', 'Clc1cc(Br)c([O-])c(=O)n1/N=c1s/c(=C\\c2ccccc2)n1Cc1ccccc1', 'Clc1ccccc1Oc1ccc(COc2ccc3c(c2)OCCCO3)cc1', 'C[C@@H](Oc1ccc(F)cc1)C(=O)NC1CC[NH+]([C@@H](C)c2ccccc2)CC1', 'Clc1ccc([C@@H](O)CCS(=O)(=O)c2ccc(Br)cc2)cc1', 'O=C(COCC1CC[NH+](c2ccc(Cl)c(F)c2)CC1)/C=C/c1ccccc1', 'O=C([C@H](CCC1CC1)c1c[nH]c2ncccc12)N1CCCC1', 'Clc1cccc(-c2ncoc2C2CCN(c3ccc(N4CCOCC4)cc3)CC2)n1', 'Clc1cccc([C@@H](Sc2ccc(Cl)cc2)c2ccc(F)cc2)c1', 'O=C([C@H](CS)c1cc(F)ccc1F)N1CCC(OC2CC2)CC1', 'OC[C@H](Cc1ccccc1)Oc1ccc(N2CCOCC2)[nH+]c1', 'Clc1cc(Cl)c(C2CC2)cc1N[C@@H]1CC[C@H](S(=O)(=O)/N=C/c2ccc3c(c2)OCO3)C1', 'Clc1ccc(Br)c(NC2(c3ccccc3)N(Cc3ccccc3)C2)c1', 'O=C([C@H](SC1NN(c2ccccc2)CC1)N1C[C@@H](CO)CC1)c1ccccc1', 'Oc1ccccc1OSC(=S)N1N=C(c2ccc(Br)cc2)C(=O)[C@H]1c1ccccc1O', 'Clc1ccccc1Oc1cc(SC[C@@H]2CCOC3=c4ccccc4=[NH+][C@@H]3[C@H]2c2ccccc2)n(C2CC2)c1', 'Oc1cc(-n2cc(-c3ccccc3)nc2-c2ccc(Br)cc2F)ccc1F', 'Clc1ccccc1Oc1cnc2scc(-c3ccccc3)c2c1', 'Nc1[nH]nc2c1C[NH+](CC(=O)N(C1CC1)[C@H]1CCS(=O)(=O)C1)CCC2', 'ClCc1ccc(C2=CCN(Cc3ccccc3)CC1)CCCC2', 'Oc1ccc(/C=N\\N=C2\\NCCS2)cc1N1CCCCCC1', 'O=[N+]([O-])c1c(Cl)cc(C(=O)c2ccc(OCc3ccc(Br)cc3)cc2)cc1F', 'O=C([C@H](Nc1nc[nH]n1)C1CCCC1)N(C1CCCCC1)[C@@H]1CCS(=O)(=O)C1', 'Clc1ccc(-c2nnnn2CC[NH2+]C2CC2)cc1', 'Oc1ccc(/N=c2/nc(-c3ccccc3)[nH]n3ncnc23)cc1O', 'ClC[C@@H]1CCC[C@@H]1COCCn1cccn1', 'Clc1ccccc1[C@H](N[C@H]1CCSc2ccc(Cl)cc21)c1ccccc1', 'Oc1cc(-n2nccc2C(=O)c2ccccc2OCc2ccc(Cl)cc2Cl)cc2c1NCCC2', 'Oc1ccc(/C=C2/N=C(CN3CCN(c4ccccc4F)CC3)N=N2)cc1', 'Clc1nnn(-c2ccccc2)c1Cn1ccnc1-c1cnccn1', 'Clc1ccc(CN[C@@H](c2ccccc2)C2CC2)cc1', 'Clc1ccccc1-c1cnn(-c2cc(F)cc3ccccc23)c1O', 'ClCc1noc(Cn2cnnn2)c1N1CCCN(c2ccccc2)CC1', 'Clc1ccccc1OCCNC(=O)[C@@H](c1ccco1)C(F)(F)F', 'N[C@@H](NC(=O)[C@H]1CCC[NH+](C2CC2)C1)C1(Cn2cncn2)CC1', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n', 'Clc1cccc2cc[nH]c12\n\n', 'Clc1cccc2cc[nH]c12\n', 'Clc1cccc2cc[nH]c12', 'Clc1cccc2cc[nH]c12', 'Clc1cccc2cc[nH]c12', 'Clc1cccc2cc[nH]c12', 'Clc1cccc2cc[nH]c12', 'Clc1ccc(-c2ccccc2)nc1CNC[C@@H](O)c1ccccc1F', 'Clc1ccc([C@@H]([C@@H](Br)c2ccccc2)C(F)(F)Cl)cc1', 'Clc1ccccc1Nc1c2onc2ncn1Cc1ccccc1', 'NS(=O)(=O)N(CC(=O)Nc1ccc(F)c(Cl)c1)[C@@H]1CCC[NH+](C2CC2)C1', 'O=C(CNc1cc2[nH]c(=O)[nH]c2cc1Br)N1CCC[C@@H]2CCC[C@H]21', 'Clc1ccc([C@@H](NCc2nccn2C2CCCCC2)c2ccccc2)cc1', 'Clc1ccc(Br)cc1OCCNC(=O)c1ccc(=O)n(CCC(F)F)c1', 'Clc1ccc(Br)c(F)c1CN(Cc1ccccc1)Cc1ccco1', 'Clc1ccccc1Cn1nnnc1[C@@H]1CNC(=O)N1C[C@H]1CC=CCC1', 'Clc1ccccc1Cc1cn(CC(=O)[O-])c2ccccc12', 'Clc1ccc(CN2NCCN(c3ccccc3C(=O)[O-])CC2)cc1', 'Clc1ccccc1Oc1ccccc1CN(Cc1ccccn1)C1CC1', 'Clc1ccc(-c2nnnn2CC[NH+]2CCCC2)cc1', 'Clc1ccc(COc2nc(CN3CCC(c4ccccc4)CC3)no2)cc1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1C[NH+]1CC[C@H](C[NH+]2CCCC2)C1', 'ClCc1cccc2c1O[C@@H](Cc1cc(Br)ccc1OC[C@@H]1CCCO1)C2', 'Clc1ccccc1NCCCO/N=C/c1ccccc1[N+](=O)[O-]', 'Clc1ccc(C[NH+](CC[C@@H]2CCCO2)Cn2cccn2)nc1', 'N[C@@H](NCc1c[nH]nc1-c1ccccc1)C(=O)N1CC[C@H]2[C@@H](CCCc3ccc(F)cc32)C1', 'Clc1ccccc1CO[C@H]1CCC[C@H]1Cc1ccc(C(F)(F)F)cc1', 'Clc1ccc([C@@H](C[C@H]2CCCO2)Sc2ccccc2)cc1', 'CC(C)(C)C(=O)Nc1c(NCc2[nH+]ccn2C2CCCCC2)cnn1-c1ccc(Cl)cc1', 'ClCc1n[nH]c2c1CCC[C@H]([NH2+]CCc1ccccc1)C2', 'Clc1ccccc1[C@H]1NC(=O)COc2c1ccc1c2cccc1OCc1ccco1', 'Oc1ccc(O[C@@H]2CCN(C(=O)Cc3c[nH]c4ccc(Cl)cc34)C2)cc1', 'O=[N+]([O-])c1ccn(CCc2ccccc2Cl)c1', 'Clc1ccccc1NCC[C@H]1CCCN1S(=O)(=O)c1c[nH]c2ccccc12', 'O[C@@H]1[C@@H](CC(=O)NNc2cnc3ccccc3n2)CCN1c1ccccc1', 'Nc1nnc(-c2cccc(F)c2)cc1CN1CCC[C@@H](c2nc3ccccc3[nH]2)C1', 'O=C(c1cn2ccccc2n1)N1CC[C@@H](CO)[C@H]1NC(=O)c1ccccc1Cl', 'N[C@@H](S/C=C/CO)c1ccc(Cl)cc1', 'CN[C@H]1CC[C@@H]([NH+]2CCC(Cc3nc(C4CC4)no3)CC2)C1', 'C[C@@H](c1ccc([S@](C)=O)cc1)N(C)C(=O)Nc1ccc(C(N)=O)c(Cl)c1', 'Clc1ccccc1-c1n[nH]c2cncc(NCC[NH+]3CCCCC3)c12', 'O=C([C@H](CCc1cccnc1)NCc1ccccc1)N1CCCC[C@H]1c1ccccc1', 'O=C(CCC1=CCCC1)N[C@H](Cc1ccccc1OC(F)F)c1ccccc1', 'O=C(NCCNc1cccc2ccccc12)c1c[nH]c(=O)[nH]c1=O', 'O[C@@H](COCc1ccc2c(c1)CCCCC2)C[NH+]1CCC[C@@H](C(NC2CC2)C(F)(F)F)C1', 'O=[N+]([O-])c1cc([C@H]2SCC(=O)N(Cc3cccs3)C2=O)ccc1OS(=O)c1ccccc1Cl', 'Clc1ccc(CCC[NH+]2CCCC[C@H]2c2ccccc2)s1', 'O=[N+]([O-])c1c(-c2c[nH]nc2Cc2ccccc2)oc2ccccc12', 'Clc1ccccc1NC(=O)N1CCN(c2ccccc2OC(F)F)CC1', 'Clc1ccc(CC[C@@H](Cl)c2cccc(Cl)c2Cl)cc1', 'Oc1ccccc1/N=C(\\N/C1=N/S=C1N1CCC[C@@H]1c1ccc(Cl)cc1)c1ccccc1', 'O=[N+]([O-])c1c(Cl)nn(C[C@@H](O)COc2cccc(OCc3ccccc3)c2)c1-c1ccc(Cl)cc1', 'COc1nc2nc(C(=O)NCCCc3ccc(F)cc3)cn2cc1[N+](=O)[O-]', 'OCc1nncc(-c2ccco2)c1C1CC[NH+](Cc2ccccc2)CC1', 'Clc1ccccc1CCOc1ccccc1C[NH+]1CCC[C@@H](O)C1', 'CN(CC1CCC([NH+]2CCCCC2)CC1)C(=O)NCCCn1cncn1', 'Clc1ccccc1NC[C@@]1(O)Cc2ccccc2C(=O)N1c1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])c2c([nH]1)C[C@@H](C(=O)N1CCCCCC1)CCC2', 'O=C(COCCCNC(=O)C1(c2ccccc2)OC1=O)c1ccccc1', 'O=C(Cc1nc([O-])c(-c2ccccc2Cl)n1C1CC1)Nc1ccccc1', 'O=C(Cc1nnnn1-c1ccccc1)N1CCO[C@H](c2ccccc2F)C1', 'Oc1cc(COC(=O)C2CCN(c3ccc(Cl)cc3)CC2)ccc1Oc1ccccc1', 'Clc1ccccc1CN(Cc1ccccc1)Cc1ccccn1', 'O=C(c1c[nH]c2nc[nH]c12)N1CCC2(CC1)N(c1ccccc1)CC[C@H]2c1ccccc1', 'Clc1ccc(Cc2n[nH]c(-c3ccco3)n2)cc1', 'Clc1ccccc1C1CCN(CC(=O)Nc2ccccc2F)CC1', 'Clc1ccccc1COCC1CCN(C(=O)c2ccco2)CC1', 'Oc1ccc(O[C@@H]2C[C@@H](C(=O)[O-])[C@H]2C(=O)Nc2ccccc2)cc1', 'CNc1[nH+]cccc1C(=O)N1CC[C@@H](c2ccccc2)C1', 'Clc1ccccc1-c1nn(CCCc2ccccc2)c2c1CCC2', 'COc1nn[nH]c1CCc1nnc2n1CC[NH+](Cc1cccc(C(F)(F)F)c1)CC2', 'O=C([C@H](c1c(Cl)cccc1Cl)C(F)F)N1CCN(Cc2ccco2)CC1', 'Nc1c(O[C@@H]2CCOC3(CCOCC3)C2)nc(N2CCOCC2)n1C1CC1', 'O=C(COCCC[C@@H]1COc2ccccc2O1)NCc1ccc2c(c1)CCCC2', 'O=C(COc1ccc(Cl)cc1)NCC1([NH+]2CCCC2)CC(C)CC1', 'CNc1ncnc(S[C@@H](C)C(=O)NC2CC2)c1C1CC1', 'Oc1ccccc1[C@@H]1C[C@@H](C[NH2+][C@H]2CCc3ccccc32)c2ccccc2S1', 'Clc1ccc(C2(OCc3ccc(-n4cnnc4)cc3)CCC2)cc1', 'O=C(CNC(=O)N1CCC[C@@H](c2nc3ccccc3[nH]2)C1)NC1CC1', 'Clc1ccccc1Oc1cnc(-c2ccccc2Cl)n1-c1ccccc1', 'O=C(c1cnn(Cc2ccccc2)n1)N1CCN(Cc2ccccc2)CC1', 'Clc1ccc(CO)n1CC12CC3CC(CC(C3)C1)C2', 'Nc1c(O[C@@H]2CCCc3ccccc32)ncnc1NC(=O)[C@@H]1C[C@@H]2c3ccccc3C(=O)N12', 'Clc1cc(/N=C/CC[NH+]2CCC[C@@H]2c2ccc(C(F)(F)F)cc2)ccc1NCC1CC1', 'Clc1ccccc1O/C(=C/c1ccc(Cl)c(Cl)c1)N1CCCCC1', 'C[C@@H](CN1CCCCC1=O)[NH2+]CC(=O)Nc1ccccc1C(F)(F)F', 'Clc1nnn(-c2ncn(-c3cccc(Cl)c3)c2-c2ccn[nH]2)n1', 'O=[N+]([O-])c1ccc2c(c1)[C@@H]1C=CCC[C@H]1[C@H](c1ccc(F)cc1)N2C1CCCCC1', 'Cc1ccc(OCc2noc([C@H]3CCCN(C(=O)[C@H]4C[C@H]3C(F)(F)F)n4C)c2C)cc1C', 'O=C(Cc1nnc(-c2ccc(Br)cc2)o1)N[C@@H](c1cscc1)c1ccccc1', 'ClC[C@@H]1CCC[NH+]1Cc1ccccc1O', 'N[C@@H](NC[C@H](c1ccco1)[NH+](C)C)c1ccccc1', 'Clc1ccc(COc2nc(Cl)ccc2Oc2ccccc2F)cc1', 'Oc1cccc(O[C@@H]2CCOc3ccc(Oc4ccccc4F)cc32)c1', 'O=C(CCc1c[nH]c2ccccc12)N[C@H]1CCC[C@H](O)C1', 'Clc1ccc(C[NH+](CC2CC2)C2CC2)cc1', 'Clc1ccc(CC[C@@H](CBr)C2CC2)cc1', 'O=C(CSCCN1CCOCC1)c1cc(Cl)c(Cl)cc1Cl', 'Clc1ccccc1N1N(C(=O)Cc2ccccc2F)CCC1', 'Clc1ccccc1Cn1c(SCc2ccccc2)nc2ccccc21', 'Oc1ccccc1CC/[NH+]=N/CCC[NH+]1CCNN(C2CCOCC2)C(=O)C1', 'O=C(c1ccsc1COCC1CC1)N1CC[NH+](Cc2ccccc2)CC1', 'O=C(/C=C\\COC1CC[NH+](Cc2cccs2)CC1)Nc1cccc2c1CCCC2', 'Clc1ccccc1-c1nc(C(=O)CCc2cccc(-c3ccccc3)c2)n[nH]1', 'O=C(C[NH+]1CCO[C@H](c2ccccc2)C1)Nc1ccc(Cl)cc1', 'Clc1ccccc1C1(C[NH2+]Cc2ccccc2)CCCC1', 'Oc1cccc(-n2cccc2C[NH+]2CCC(N3CCCC3)CC2)c1', 'Clc1ccccc1Cc1nc2ccccc2nc1NC[C@H]1COCCO1', 'Clc1ccccc1C(=O)Nc1ccc(Cn2nncc2C2CC2)cc1', 'Clc1ccc(Cc2noc(C[C@@H]3CCCN3C(=O)c3ccccc3)n2)cc1', 'CN=C/[NH2+]Cc1ccc(-c2ccccc2)[nH]c1=O', 'Clc1nnnc(-n2cc(N3CCOCC3)nc2-c2ccccc2F)c1C1CC1', 'Clc1ccccc1Cn1c2ccccc2c2ncnn21', 'C[C@@H](Oc1ccc(Cl)cc1Cl)C(=O)Nc1ccc(C(=O)O[C@H](C)C(F)(F)F)cc1', 'O=[N+]([O-])c1ccnc2ccc([N-]S(=O)(=O)c3cc(F)ccc3F)cc1c1ccccc12', 'Oc1ccc(/N=c2/[nH]ccn2CCOc2ccccc2)cc1', 'Clc1ccccc1OCC1CCCC1', 'OC[C@@H](NS(=O)(=O)c1ccc(Cl)cc1Cl)c1ccc(Cl)cc1', 'Clc1ccccc1[C@H]1CN(C(=O)Cn2nnnc2-c2ccccc2)CCO1', 'Clc1ccc([C@@H](Br)Cc2cccnc2)cc1', 'Clc1ccc(Br)c([C@H]2CCC[NH+]2Cc2ccccc2)c1', 'ClCc1n[nH]c2c(n1Cc1ccccc1)c(=O)n(CC(C(C)CO)[C@@H]2CNF)c1ccccc1', 'Clc1ccc(Cc2noc(C[NH+]3CCN(c4ccc(Cl)cc4)CC3)n2)cc1', 'Clc1ccc(-c2n[nH]c(C[NH2+]c3ccc(N4CCOCC4)cc3)n2)cc1', 'Clc1ccccc1Oc1cc(-n2cccn2)nc2cccc(F)c12', 'O=C([O-])/C(C(=O)c1ccccc1)NCCc1ccccc1', 'C[C@@H](C(=O)Nc1ccc(N2CCCCC2=O)cc1)n1cc(-c2ccc3ccccc3c2)nn1', 'OC1(Sc2ccccc2OCC2CCCCC2)CCCCCCC1', 'Clc1ccc(Br)cc1NCCOCCOCn1ccccc1=O', 'Clc1ccccc1OCCCn1ccc2ccccc21', 'O=C(CNc1nn2cnnc2c2c1CCCC2)NNC(=O)c1ccc(Br)cc1', 'Clc1ccccc1-c1cncnc1NC[C@@H]1C[C@H]1c1ccccc1', 'O=C(c1c[nH]c2cc([N+](=O)[O-])ccc12)N1CCN(C(=S)NCC2CC2)CC1', 'Oc1ccccc1[C@@H](Cc1ccc(Br)s1)N1C[C@H]2CCCC[C@@H]2C1', 'O=C(COc1cc(C2[C@H](C(F)(F)F)N=N2)[nH]n1)N1CCN(c2ccc(F)cc2)CC1', 'OCc1nnn2c1N[C@]1(C=CCC1)[C@H]2CC(=O)NC1CCCCCC1', 'NS(=O)(=O)Nc1ccc([N+](=O)[O-])c(NCC[NH+]2CCCCCCC2)c1', 'ClCc1ccn(C[C@H](O)c2ccccc2)c1', 'ClCc1ccn(C[C@H](O)c2ccccc2)c1', 'O=C(COc1cc(NC(=O)NC2(c3cccs3)CCOCC2)on1)N1CCCCC1', 'Clc1ccc(C[NH2+][C@@H]2CCCC[C@H]2CN2CCOCC2)cc1', 'Clc1ccccc1COC1CCC(CNC(=O)CCOc2ccccc2)CC1', 'Clc1ccccc1Oc1cncc(N2CCC[C@H](Oc3ccccc3F)C2)n1', 'O=C([O-])c1c(N2CC(c3ccccc3)n3nc(C4CCCCC4)ncn23)coc1-c1noc2c1CCCC2', 'O=C(/C=C1CCN(C(=O)CCc2ccccc2)N=N1)Nc1ccccc1F', 'Clc1ccc(C[NH2+]CCNc2ccc(Br)cc2)cc1', 'N[C@@H](C1CCC(c2cccs2)CC1)N1CCN(c2ccccc2F)CC1', 'O=C(c1ccsc1)[C@@H]1CN(C[C@@H]2CCCO2)CCN1C[C@H]1CCCO1', 'Clc1ccc([C@@H]([NH2+]C[C@H]2CCCO2)N2Cc3ccccc3C2)nc1Cl', 'O=C(CCCn1cnnn1)N1CCN(Cc2ccc(F)c(F)c2)CC1', 'CCn1nnc(CN2CCN(C(=O)Cc3ccc(F)cc3F)CC2)n1', 'Clc1ccccc1-c1nn(CN2CCc3ccccc32)c2c1CCCC2', 'O=C([C@H]1CCCN1C(=O)c1cnc2ccccc2c1)N1CCN(c2ccccc2C(F)(F)F)CC1', 'Clc1ccccc1Oc1cc(F)c(N2CCCC2)c(I)c1', 'Clc1ccc(-c2cccs2)c2nnc(-c3ccccc3Cl)nc12', 'O=C(c1cn(Cc2ccco2)nc1-c1ccco1)c1ccc(Br)cc1', 'O=C(c1cccc2ncccc12)N1CCC(Cn2cc[nH+]c2-c2cccnc2)CC1', 'Clc1ccc(C2=CC[NH+](Cc3ccccc3)CC2)cc1', 'Clc1ccccc1C[NH+](C[C@@H]1CCCO1)[C@H]1CC[C@](O)(COCC(F)F)C1', 'Clc1cc(C[C@@H]2CCCN(S(=O)(=O)C[C@@H]3C=C(CN4CCOCC4)CO3)[C@@H]2c2ccccc2)ncn1', 'Nc1c(CSc2nnc(-c3ccccc3)cn2)oc2c1CN(Cc1ccc(F)cc1)CC2', 'Nc1c(/C=N/O[C@H]2CCCc3ccccc32)ncnc1NCc1ccc[nH]1', 'C[C@@H]([NH2+]C1CCC(N2CCOCC2)CC1)c1ncnc2c1CCCCC2', 'ClCc1ccnn1Cc1ccccc1Oc1ccccc1F', 'O=C(COc1cc(Cl)ccc1Cl)N1CCc2ccccc21', 'Clc1ccc(-c2nnnn2CC(=O)N2Cc3ccccc32)cc1', 'Clc1ccccc1[C@H]1C[C@@H](c2cc(F)cc(Br)c2)[NH+](CC2CC2)C1', 'Clc1ccc(Br)cc1C[NH+](Cc1cc(O)ccc1C)[C@H]1CCSC1', 'O=C(Cc1cc2c(cc1Br)OCO2)N1CCN(Cc2ccccc2)CC1', 'Oc1ccccc1[C@H](c1ccccc1)N1CCC(C[NH2+][C@@H](CO)C(F)(F)F)CC1', 'O=C(COCCOC1CCCCCC1)Nc1ccccc1C(F)(F)F', 'NS(=O)(=O)CCNC(=O)[C@@H]1OCC(=O)N(C)[C@H]1c1ccc(Cl)cc1', 'ClCc1ccccc1-n1cccc1', 'O=C([C@H](c1c(Cl)cccc1Cl)N1CCCC1)N(C1CCCCC1)S(C)(=O)[O-]', 'OC[C@@H]([NH2+]Cc1ccncc1)Cc1ccccc1', 'Oc1ccc(F)cc1CC1CCN(C(Cc2ccccc2)Cc2ccco2)CC1', 'Clc1ccccc1C1(OCCN2CCOCC2)CC1', 'Cc1c(-n2cccc2)c2ccccc2n1CC(=O)N[C@H]1CCC[C@@H]1[S@@](=O)Cc1ccccc1', 'Clc1ccccc1Oc1cnc2ccccc2c1OC[C@H]1CCCO1', 'O=C(/C=C/c1c(NC2CC2)oc2ccccc12)NNC1CCCC1', 'Clc1ccc(Br)c(S[C@H]2CCOC3(CCCCC3)C2)c1', 'O=C(Cc1n[nH]c2c1CC[NH2+]CC2)N1CCc2ccccc21', 'Oc1ccc(Oc2ccc(F)cc2)cc1CN1CCN(Cc2ccccc2)CC1', 'O=C(/C(Cc1ccco1)c1ccc(F)c(Cl)c1)Cn1c[nH+]ccc1=O', 'O=C(/C=C/c1cccc([N-]S(=O)(=O)c2ccc(Cl)cc2)c1)N(Cc1ccccc1)C1CC1', 'Clc1nnnn1CC[NH2+]Cc1ccccc1OC(F)F', 'Clc1nnnc(OCCC[NH2+]Cc2ccccc2)c1[N+](=O)[O-]', 'Clc1ccccc1Cn1ncc2c1CCN(C(=O)[C@@H]1COc3ccccc3C1)C2', 'Clc1ccccc1OCc1ccncc1COCC1CC[NH+](C)CC1', 'O=C(c1c[nH]c2ncn(-c3ccccc3)c12)c1ccccc1', 'O=C(c1cccc(-c2noc(-c3cc(F)ccc3F)n2)c1)[NH+]1CC[C@H]([NH+]2CCCC2)C1', 'Clc1ccc(COCc2ccccc2)[nH+]c1', 'Clc1ccc(COc2nc([N+](=O)[O-])ccc2N2CCCCC2)cc1', 'Clc1ccc(Br)c(F)c1COC[C@@H]1Cc2ccccc2S1', 'N=C1[C@H]2CC3=CC=CC(C)(C)CC[C@@H]3C[C@@H](SCC(=O)Nc3ccccc3F)[C@@H]12', 'Clc1ccc([C@@H](CCCl)Nc2ccccc2F)cc1', 'O=C(CSC(c1ccccc1)c1ccc(Cl)cc1)Nc1ccccc1C(=O)N1CCCC1', 'COc1ncnc(N)c1OC[C@@H]1C[C@]12CCc1ccccc12', 'Clc1ccccc1Oc1cc(-c2nnc(-c3cccs3)o2)ccc1F', 'O=[N+]([O-])c1c(-c2nc(C3CC[NH2+]CC3)co2)cc(O)c2c1CCN(C(=O)[C@@H]1COc3ccccc3O1)C2', 'O=C([O-])c1nn(-c2ccccc2)cc1N1CCN(Cc2ccccc2)CC1', 'CC[C@@H]1O[C@H](C(=O)N2CCc3c([nH]c4ccccc34)N2C)C(=O)N1', 'Clc1ccccc1CNc1n[nH]nc1Cc1ccc(Br)cc1', 'O=C(c1c[nH]c(=O)[nH]c1=O)N(Cc1ccccc1Cl)C(F)(F)F', 'O=C(CCN[C@@H]1COc2ccccc21)Nc1ccc2c3c(cccc13)CS2', 'O=C(CC[NH+]1CCC[C@@H]1c1ccco1)Nc1ccccc1[N+](=O)[O-]', 'Clc1ccc(CN[C@@H](C(=O)c2ccccc2)c2ccccc2)cc1', 'Clc1ccc(C[NH2+][C@H](c2ccccc2)c2cccnc2)cc1', 'Clc1cccc2ccc(Nc3ccccc3C(=O)OC[C@H]3CCCO3)c2c1', 'OC[C@@H](NCc1ccc(OCCO)cc1)c1ccccc1', 'OC[C@@H](N[C@@H]1CCN(CC2CC2)C1)c1nc2ccccc2[nH]1', 'Clc1ccccc1[C@H]1C[C@@H](CC(=O)Nc2cccc(C(F)(F)F)c2)N(Cc2ccc(F)cc2)C1', 'Oc1ccc(Oc2ccc(NCc3nc4ccccc4[nH]3)cc2)c(Cl)c1', 'Clc1cc(Nc2nc3ccccc3o2)ccc1OCc1ccccc1Cl', 'Clc1cccc(F)c1CCNC[C@@H]1CCc2ccccc21', 'O=C(COc1cccc2ccccc12)Nc1ccc(N2CCOCC2)cc1', 'Nc1[nH]ncc1-c1nnc([C@H]2CCCN(C(=O)CCc3ccc(Cl)cc3)C2)n1Cc1ccccc1', 'Clc1nnn(C[C@@H](CO)Oc2ccccc2)c1CN1CCOCC1', 'Clc1ccc(Cc2nnn(CC(=O)NCCN3CCOCC3)c2Cl)cn1', 'Clc1ccc(Cc2ncoc2CNc2ccc(-n3cccc3)o2)cc1', 'NS(=O)(=O)/C=c1/C(C#N)c2cc(Cl)c(C(=O)N3CCOCC3)cc2nc1/C=N/C(=O)c1ccccc1O', 'Clc1ccccc1O/N=[N+]=NC(=O)CN1CCN(c2cnccc2)CC1', 'Clc1ccc(-c2n[nH]cc2CN2CC[C@@H](Nc3ccccc3F)C2)cc1', 'Clc1nnnn1c1[nH]cnc1C[C@@H]2CCC[NH+](Cc1ccccc1Cl)C2', 'Clc1ccc(-c2nnnn2C[C@@H]2CC3(CCOCC3)C2)cc1', 'Clc1cccc(Br)c1CC1OCCC[C@@H]1C(=O)[O-]', 'Clc1ccccc1-c1n[nH]cc1C[NH2+][C@@H]1CCc2ccccc21', 'Clc1ccc(CC[C@H]2CCCN(c3ncccn3)C2)cc1', 'Clc1ccc(-c2cc(NCc3ccco3)ccc2C[NH+]2CCCC2)cc1', 'Clc1ccc(-c2nonc2)nc1COCc1ccccc1[N+](=O)[O-]', 'Clc1ccc(Br)c([C@@H](CO)Nc2ccc(F)cc2)n1', 'O=C(c1c[nH]c2ncn(CC(=O)NCCc3ccccc3)c12)N1CCOCC1', 'O[C@@H]1c2ccccc2C[C@@H]1CC[NH+]1CCCC[C@H]1C1OCCO1', 'Clc1ccc(CO)nc1NCc1ccc(N2CCC2)nc1', 'Clc1ccc(-c2n[nH]c3cc(C(=O)NCC[NH+]4CCCCCC4)ccc23)cc1', 'Clc1ccc(Br)c(CCC(=O)N2[C@H](c3ccccc3)OC2(C)O)c1', 'Clc1ccc(-c2csc3scc(-c4ccc(Cl)c(F)c4)c3c2=O)cc1', 'Clc1ccc(-c2ccccc2[N-]C[C@H](O)C[NH+]2CCCC2)cc1', 'Clc1ccccc1-c1cscc1-c1cc(=O)[nH]c2ccccc12', 'O=[N+]([O-])c1cc(F)cc(CN2CCOC[C@H]2Cn2cnnn2)c1', 'Clc1ccc(Br)c(F)c1C[NH+]1CCC[C@H](c2nncn2C2CC2)C1', 'Clc1ccc(COC[C@H]([NH2+]Cc2ncc[nH]2)c2ccccc2)cc1', 'Nc1c(/N=N/NC(=O)c2ccnc(OC3CCC3)c2)[nH]c2c1CCC[C@H]2O', 'O=C(CN(CCO)c1ccccc1)Nc1ccc(Cl)cc1', 'OC[C@@]1(NC2CC2)Cc2ccccc2C1', 'Clc1ccc(CNc2sc3c(n2)CN(Cc2ccc4c(c2)OCO4)CC3)cc1', 'O=C(COCCNC(=O)N(C1CC1)c1ccccc1)OC[C@@H]1CCCO1', 'Clc1cc(/C=C/N2CCN(c3cccc([N+](=O)[O-])c3)CC2)c[nH]1', 'Clc1ccc(CNC(=O)N2CCCCC[C@@H]2c2ccccc2)o1', 'Oc1cccc(-c2cc[nH]c2)c1CCC(=O)Nc1ccccc1Cl', 'CCn1c(C2CCCCC2)ncc2c(C)nn(CCC(=O)Nc3cccc(F)c3Cl)cc21', 'Clc1ccccc1C(C)CCOc1c(Cl)cccc1Cl', 'Clc1ccccc1O/C=C/CN1CCCSCC1', 'N=CCNc1c(-c2[nH+]c3ccccc3[nH]2)c(=O)oc2ccccc12', 'Oc1ccccc1[C@@H]([NH2+]Cc1ccco1)c1ccccn1', 'O=C([C@H](SCO)c1ccc(Br)cc1)N1CCN(Cc2ccccc2)CC1', 'O=C([C@H](C(F)(F)F)[NH+]1CCN(CCO)CC1)Nc1ccccc1C(F)(F)F', 'COc1nn(C)cc1CN[C@@H](C)c1ncc(-c2ccccc2)n1C', 'O=C(c1cnn(C[C@@H]2CCCO2)c1)N1CCN(c2ccc3c(c2)OCO3)CC1', 'O[C@@H]1c2ccccc2C(=O)N1CCOc1ccc(Br)cc1', 'CCSc1ccc(/C=C/C(=O)[O-])cc1', 'O=C(Cc1non2cccc12)N1CCN(CC(=O)/N=C\\c2ccccc2Cl)CC1', 'O=C(CSCC1=C2C=CC=C2C[C@H]1Cc1ccccc1)OCc1ccccc1', 'O=C(c1c[nH]c2nc[nH]c12)N1CCC[C@H]1C[C@H](O)c1ccccc1', 'O=C(/C=C1/NNc2ccccc2N1)N1CCC(OC[C@H]2CCCCO2)CC1', 'Clc1ccc(Br)c(S[C@H]2CCO[C@]3(CCSC3)C2)c1', 'Clc1ccccc1CSC[C@@H]1CCCCC[C@@H]1C(=O)[O-]', 'Clc1cccc2c1CCC[C@H]2[NH2+][C@@H](c1cccc(CC(F)F)c1)[C@H]1CCS(=O)(=O)C1', 'ClC(F)/C=C/C=C/C=C/C=C/c1ccc(Cl)c(-c2nc(C3CC3)cs2)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(C(F)(F)F)c1C[NH2+]Cc1ccccc1', 'CCCCN(Cc1ccco1)C(=O)Nc1ccc(Cl)cn1', 'O=C(C1CC=C(c2ccc(Br)cc2)SC1)N1CCN(C(=O)COC(F)(F)F)CC1', 'O=C([O-])c1[nH]ncc1Cn1c(CO)nc2ccc(-c3ccccc3)nc21', 'OC[C@@]1(OC2CCCCC2)C[C@H]1c1cnc[nH]1', 'CCCNC(=O)CC1CCN(C(=O)c2cccc3[nH]cnc23)CC1', 'Clc1ccccc1OCCC[NH2+]Cc1ccc(OC(F)F)cc1', 'Oc1ccc([C@H](N[C@@H](Cn2cccn2)C2CC2)N2CCCC2=O)cc1', 'O=C(/C=C\\c1c(OC2CCCC2)cc(O)c2c1CSCO2)N1CCCCC1', 'Clc1ccccc1Oc1cnc2nnc(-c3ccc(C(F)(F)F)cc3)n2c1', 'Clc1cccc2ccc(NCCC[NH+]3CCCC3)nc12', 'Clc1nncnc1C[C@H]1CCC[NH+](CCc2c[nH]c3ccccc23)C1', 'Clc1ccccc1CN(Cc1ccsc1)Cc1ccc(F)cc1', 'O=C(COc1cc(N(C(C)C)[C@H]2CCCC2=O)oc1C#N)N[C@@H](c1ccccc1)c1ccccc1', 'Clc1nnnc(-n2cnnc2COC(=O)c2cccc(OCc3ccccc3)c2)c1Cl', 'Clc1cccc2cccnc1n2CC[NH2+]Cc1ccccc1', 'CN=C(OC)CC[NH+]1CCN(CCc2ccc(Cl)cc2)CC1', 'Clc1ccccc1C[NH+](Cc1ccccc1Cl)[C@@H]1CCN(C(=O)OC(F)(F)F)C1', 'Clc1nnnn1CCN1CCOCC[C@@H]1C[NH+]1CCCC1', 'Clc1ccc(-c2nonc2CN2CCN(c3ccccc3)CC2)cc1', 'Clc1ccccc1CSC/C=C/C(=O)[O-]', 'O=C(COc1cc(F)c(Br)cc1F)N1CCC[C@@H]1c1ccccc1', 'Clc1ccc(-c2nonc2N2CCC[C@H](C(=O)Nc3ccc4c(c3)CCC4)C2)cc1', 'Clc1nnc(/C=C/c2ccccc2)n1/N=C/c1ccco1', 'Clc1ccc(C(F)(F)F)nc1OCCCCCO', 'Clc1ccc(-c2ccccc2)nc1CN[C@H]1CCCC[C@H]1CO', 'CCNC(=O)N[C@H](C)c1nc2ccccc2n1CCN1CC[NH+](C)CC1', 'Clc1ccccc1Oc1cc(-c2noc(-c3cccnc3)n2)ncn1', 'O=C(c1cnn2cc[nH]c12)OCc1cnn(-c2ccc(Cl)cc2)c1', 'ClC(=S)O/C(=C/c1ccc(N2CCCC2)cc1)c1ccc(F)cc1', 'Clc1cccc(Br)c1NCCCOc1ccccc1F', 'Clc1cc(Br)cnc1N1CCO[C@@H](NC2CC2)C1', 'O=C(C1=c2ccccc2=[NH+][C@@H]1C[NH+](C)C(C)C)[C@@H]1CC(=O)N(c2ccccc2F)C1', 'Clc1ccc(Br)cc1/N=C/C(=CC1=c([O-])c([O-])c(Cl)c1=O)N1CCCCCC1', 'ClC(c1cc(C2CC2)nn1-c1ccc(Br)cc1)c1ccc(Cl)c([N+](=O)[O-])c1', 'ClCc1n[nH]c(=O)n1CCOc1ccccc1C(F)F', 'Oc1ccccc1[C@H](Nc1cccnc1OC[C@H]1CCCO1)c1ccc(Cl)cc1', 'ClC[C@H]1CCCC[C@@H]1c1ccccn1', 'Clc1cc(/C=Cc2ccc(COC3CCCC3)cc2)no1', 'Oc1cccc(Br)c1C[NH+]1CCC[C@@H]1c1cncn1C1CC1', 'Clc1ccccc1OC1CN(C(=O)CNc2cccc(F)c2)C1', 'COc1ncnc(NC(=O)[C@@H](C)n2c(=O)n(Cc3ccc(F)cc3)c(=O)c3ccccc32)c1C', 'O=[N+]([O-])c1ccncc1C[NH2+][C@@H]1CCN(C[C@@H]2CCCO2)C1', 'O=[N+]([O-])c1c(OCCN2CCCCCC2)cnn1C(Cl)F', 'N=CC(=O)NC1CC[NH+](CCCn2ncc(C(F)F)n2)CC1', 'COc1nnn(CCCNC(=O)[C@H]2CCO[C@H]2C)c1[N+](=O)[O-]', 'O=C(CCc1cc(-c2ccccc2)n[nH]1)N1CCCCCCC1', 'Clc1ccccc1CCc1cccc(C(=O)N2CCC[C@@H]2c2cccc(F)c2)c1', 'ClC[C@@H]1Oc2ccccc2C[NH+]1CCC1CC1', 'NS(=O)(=O)C[C@H]([NH2+]Cc1ccccn1)N1CC[NH+](C2CC2)CC1', 'O=C(Cc1n[nH]nc1C(F)(F)F)Nc1cnn(Cc2ccccc2)c1', 'Clc1cccc2c1C[NH+](C[C@@H]1C(=O)NCC[NH+]1Cc1ccccc1Cl)[C@@H]1CC=C[C@H]21', 'O=C([C@H](SC1[NH+]=c2ccccc2=[NH+]1)c1ccc(Cl)cc1Cl)NC1CC1', 'OCc1n[nH]cc1O[C@@H](c1ccccc1)[NH2+]CCN1CCCC1', 'C[C@@H](C[NH2+][C@@H]1CCN(c2ccccc2F)C1)N1CCOCC1', 'O=C([O-])/C(CO)CCn1nnc2ccccc21', 'O=C(Cc1ccccc1)NCc1nncn1C1CC1', 'Oc1cc(-n2nc3ccccc3c2[N+](=O)[O-])cc(OCc2ccccc2)c1', 'Clc1ccc(Br)cc1[C@@H]([NH2+]Cc1ccccc1)C1CC1', 'Oc1cccc(CN(CCN2CCOCC2)C(=S)N2CCCCCC2)c1', 'COc1nc2nc(CCc3ccc(F)cc3)nn2c(=O)n1Cc1ccccc1', 'Clc1ccccc1OCCn1c(=O)[nH]c2ccccc21', 'O=C(CNC(=O)Cc1ccccn1)NCC1(c2ccc(F)cc2)CCC1', 'COc1nnnn1CCC(=O)N(Cc1ccc[nH]1)[C@@H](C)c1ccccc1', 'O=C(c1c[nH]cn1)[C@H]1CCCN1C(=O)Cc1ccco1', 'Clc1ccccc1-c1ncc(COc2ccccc2)[nH]1', 'Clc1ccc([C@@H]([NH2+][C@@H]2CCOC2)c2ccc(Cl)cc2)cc1', 'O=C(COCc1noc2ccccc12)Nc1ccc(Br)cc1', 'O=C(CNCc1nc2ccccc2c(=O)[nH]1)c1ccc(Br)cc1', 'O[C@@H]([C@@H]1CCC[NH+]1Cc1c(Cl)cccc1Cl)NCc1ccccc1', 'Clc1ccc(Br)cc1O[C@H]1CC[C@@H]([NH+]2CCCC2)C1', 'Clc1ccc(Br)cc1O[C@@H]1CC[C@@H]([NH+]2CCCC2)C1', 'Clc1ccc(Br)c(OC2CC[NH+](C3CCCC3)CC2)c1', 'OCc1n[nH]c(NNC(=O)N[C@@H](c2ccccc2)C2CCCCC2)n1', 'Clc1ccccc1Cn1cnc2cc(F)ccc21', 'Oc1ccccc1OCCN(C[C@@H]1CCCO1)c1sccc1C1CC1', 'O=C(c1cn(C[C@@H]2CCCO2)cn1)c1ccccc1', 'Clc1ccc(CO)n1-c1ccccc1', 'O=C(/C(Cc1ccccc1)c1ccc(F)cc1)c1c(F)cccc1F', 'COCc1c(N)[nH+]c(C)n1CC[NH+]1CCC[C@H]1CC', 'Oc1ccc(F)c(F)c1C[NH+]1CCC(OCc2ccccc2)CC1', 'O=C(CNc1ccc([N-]S(=O)(=O)c2ccccc2)c([N+](=O)[O-])c1)Nc1ccc(Cl)cc1Cl', 'O=C(c1ccsc1CO)C(=O)NC[C@@H](c1cc2ccccc2[nH]c1=O)N1CCCC1', 'O=C(COCCOc1ccc(Br)cc1)Nc1nc([O-])c(Cl)c2[nH]ccc12', 'Clc1ccccc1O/N=C(\\[O-])CCCn1nc2ccccn2c1=O', 'ClCc1cnn(CCC(=O)c2ccc(Cl)cc2)c1-n1cccn1', 'Oc1cccc(Br)c1OC[C@@H]1CN2CCC[C@@H]2CO1', 'Clc1nnc(CCC[C@H]2CCCO2)nc1C1CC1', 'O=[N+]([O-])c1ccn(CN2CC[C@@H](c3ccccc3)C2)c1', 'Clc1ccccc1C(C1CC1)[C@H]1Sc2ccc(Cl)cc2N1c1ccccc1', 'OCc1nn(-c2ccc(F)cc2)c2c1C[C@@H](NCc1ccco1)C2', 'Clc1ccc(Br)cc1[C@@H](CO)N[C@@H](CO)c1ccco1', 'O=C(/C(Cc1ccccc1)Cn1cccn1)c1ccc(Cl)cc1', 'O=C(Cc1nonc1CO)NCCc1ccccc1', 'O=C(COCCO)O[C@@H](c1ccccn1)c1ccccc1', 'O=C(c1cccc(C2CCOCC2)c1)N1CCC(/C(Cl)c2ccccc2)CC1', 'Clc1ccc(CN2C[C@H](Cc3ccccc3)O[C@H](c3ccccc3)C2)cc1Cl', 'O=C(Cc1coc(-c2ccccc2)n1)NCc1ccccc1', 'O=C(/C(CNc1ccccc1)c1ccccc1)c1ccncc1', 'ClC(c1ccoc1C(F)(F)F)[C@@H]1C[C@@H](I)C=CC[C@H]1C(=O)[O-]', 'Clc1cc(Cl)c2c(n1)CCC[C@@H]2[NH2+][C@@H](c1ccccc1)c1cccc(N2CCOCC2)c1', 'Nc1[nH]ncc1N[C@H]1CCN=C1Cc1nnc2n1CCC2', 'Clc1ccccc1Oc1cnccc1C(=O)N1CCCC[C@@H]1C#CCCl', 'Clc1ccccc1[C@H]([NH3+])c1cnn2c1CN(Cc1ccccc1Cl)C2', 'O=C(/C=C1C(=O)N(c2ccc(Cl)cc2)c2ccccc21)c1ccccc1', 'Oc1cc(-n2cccc2)ccc1NC1CC[NH+](Cc2ccco2)CC1', 'Clc1cccc2ncnc(N3CCO[C@H]4CCCC[C@@H]43)c12', 'O=[N+]([O-])c1ccn(Cn2nnnc2-c2ccccc2)c1', 'O=C(COCc1nc2ccccc2c(=O)[nH]1)Nc1nc(C(F)(F)F)n[nH]1', 'Oc1ccccc1C[C@H]1CCC[NH+](C2C[NH+](c3nc(OCc4ccccc4)cs3)C2)C1', 'O[C@@H](CCc1ccccc1)C(=O)Nc1ccccc1C(=O)[O-]', 'COC[C@H]1CCC[NH+]([C@@H](C)c2[nH+]ccn2CC)C1', 'Oc1cc(OC2CC[NH+](Cc3ccccc3F)CC2)cc(SCC(=O)NC2CCC2)c1', 'Oc1ccccc1Cc1ccccc1OCc1ccccc1Cl', 'Clc1ccc(Br)cc1-c1ccnc(N2CCN(Cc3ccccc3Cl)CC2)c1', 'Clc1ccc(Br)cc1-c1ccc2c(c1)nc(Cn1cccn1)n2C1CC1', 'Clc1ccc(Br)cc1/N=C/c1cc(C(F)(F)F)ccc1SC1CC1', 'O=C(COc1ccc(Br)cc1)NCC[NH+]1CCCCC1', 'Clc1ccc(CN[C@@H](Cn2cnc3ccccc32)N2CCCC2)cc1', 'O=C([C@H](c1c(Cl)cc([O-])cn1)N1CCCCC1)N1CCN(Cc2ccco2)CC1', 'Oc1ccc(/C=N/N2CCN(c3ccccc3)CC2)cc1', 'Clc1ccccc1OCCSc1ccccc1F', 'CNC(=O)NC(=S)N(CCCO)Cc1ccccc1', 'Clc1nnnc(Oc2ccccc2)c1N1CCN(c2ccccc2)CC1', 'Clc1ccc(C[NH2+][C@@H]2CCO[C@]2(c2ccccc2)c2ccco2)cc1', 'ClCc1nnn2c1N[C@@H](c1ccccc1-n1nnnc1COCCO)C(=O)CCC2', 'CNc1[nH+]ccc(C[NH+]2[C@@H](C)C[C@H]([NH2+]C)CC[C@@H]2C)n1', 'Clc1ccc(Cc2ccc(CCn3nc(-c4ccccc4O)o3)o2)cc1', 'Clc1cccc2c(-c3cccc(Br)c3)cnn12', 'O=C([C@H](Nc1ncccn1)C1CC1)NC1(CC(F)(F)F)CCCC1', 'Clc1ccccc1[C@H](NS(=O)(=O)c1ccc(Cl)cc1)c1cccc2ccccc12', 'O=C([C@H](Sc1ccc(Cl)cc1)C(=O)NCc1ccccc1)NC1CCCC1', 'O=[N+]([O-])c1ccncc1O[C@H](Nc1ccc(Br)cc1)C(=O)NC1CCCCC1', 'OCc1ccc(N2CCC[C@H](c3nn(CC(F)F)c4ccccc34)C2)c(Cl)c1', 'Clc1ccc(Br)c(F)c1CCC(=O)NCCc1ncn[nH]1', 'Clc1ccc([C@@H](NCc2ccccn2)Cc2cccnc2)cc1', 'Oc1ccc([N+](=O)[O-])c(I)c1OC[C@H](O)c1ccc(F)cc1', 'O=C(CNCc1nc[nH]n1)N1CC[NH+]([C@H]2CCOC2)CC1', 'Clc1ccsc1[C@H]([NH2+][C@@H]1CC[C@@H]([NH+](C)C)C1)c1ccccc1', 'Oc1ccccc1OSC1(CC2CC2)CCOCC1', 'O=C(COc1cc(Cl)ccc1C(=O)[O-])N1CCc2ccccc2C1', 'COCCOc1ccc(CN2CCC[NH2+]C3(CCCCC3)C2)cc1', 'Clc1cccc([S@@]CC(F)F)c1', 'O[C@@H]1Cc2ccccc2CN1C(=O)COCc1ccccc1', 'ClC([O-])CC[C@@H]1CCCC[C@@H]1n1ccnc1', 'Clc1ccc(Br)c(/N=C/C=C/C=C2\\C(=O)N(Cc3ccccc3)S/C2=N\\c2ccco2)c1', 'O=[N+]([O-])c1ccn(CC[C@@H](O)COc2ccccc2)n1', 'O=C(COCc1nc[nH]n1)NCCc1ccccn1', 'CN(c1ccnc(Cl)c1)c1ccc(Cl)cc1OC[C@@H]1Cc2ccccc2O1', 'O=[N+]([O-])c1ccc[nH+]c1N1CCC(NC2CCOCC2)CC1', 'Clc1ccc(CO)n1[C@@H]1CC(=O)N(Cc2ccco2)C1=O', 'Clc1ccc([C@@H]([NH2+]CC(=O)c2ccc(F)cc2)C2CCCC2)cc1', 'Clc1ccc(Br)cc1OCCS(=O)(=O)c1ccc(F)c(F)c1F', 'Clc1ccccc1C(=O)OCCNC(=O)c1ccc(F)cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1C[C@@H]1CCC[NH+](C2CCCC2)C1', 'O=C(c1cnc(N2CCN(CC3CC3)CC2)cn1)N1CCOCC1', 'O=C(CNC(=O)CS[C@@H]1N=NC=C1O)NC[C@@H](CO)c1ccccc1', 'O=C(c1cccn1Cc1ccccc1)N1CCc2[nH]c[nH+]c2C1', 'Oc1ccc(/C=N\\N[C@@H]2CCC[C@@H]2C[C@@H]2CCOC2)cc1', 'C[C@@H]([C@@H](O)c1ccc2c(c1)CCCC2)[NH+](C)Cc1ccccc1O', 'O=C(Cc1cnn(Cc2ccccc2)c1)NC[C@@H](CCO)[NH+](C)[C@@H](C)c1ccccc1', 'O=C(CNS(=O)(=O)CC1CCC1)Nc1ccc(N2CCOCC2)nc1', 'CNS(=O)(=O)N1CCN(C(=O)[C@@H](C)Cc2c(C)noc2C)CC1', 'Clc1ccccc1CNC1=N/C(=C/c2ccncc2)N1c1ccccc1', 'Oc1ccc([C@H]([NH2+]C[C@H]2CCCO2)[C@@H]2CCSc3ccccc32)cc1', 'O=C([O-])[C@@H](Cc1ccccc1)N1CCc2ccccc21', 'N=Cc1nc2ccc(N[C@H]3CC[C@@H]([NH+](CC)C)c3)cn2c1[O-]', 'ClC[C@H]1C[NH+](Cc2ccccc2)CC1', 'O=[N+]([O-])c1c(F)cc(N2CCN(C(=O)COc3ccccc3)CC2)cc1O', 'Clc1ccc(C2(CO)CC2C[NH2+]Cc2ccsc2)cc1', 'Oc1ccc(O[C@H]2CS(=O)(=O)C[C@@H]2C[NH2+]C2CC2)cc1F', 'Clc1ccc(Br)cc1-n1cnc2c(F)cccc21', 'O=C([C@H](Sc1ccccn1)C1CC1)N1CCN(c2ccccc2F)CC1', 'O=C([C@H](Sc1n[nH]c2ccccc12)c1ccccc1)N(Cc1ccc(Cl)cc1)C1CC1', 'O=C(CSCNOCc1ccccc1)N1CCCC1', 'Oc1cc(Cn2c(=O)oc(C(=O)NCc3ccccc3)c2C2CC2)ccc1[O-]', 'Oc1cccc(Oc2nc(-c3cccs3)nc3ccccc23)c1', 'Cc1cn([C@H](C)CNC(=O)C[C@H](O)c2ccoc2)n1C', 'Clc1ccc([C@@H]([O-])CCCNc2ccccc2F)o1', 'O=C(CNCc1c(Cl)cccc1Cl)OCC(=O)Oc1cccc(Cl)c1Cl', 'O=C(/N=C(\\[O-])[C@H]1CC(c2cccc([N+](=O)[O-])c2[N+](=O)[O-])C1)OCc1ccc(Cl)cc1', 'O=C(c1ccsc1)N(C[C@@H](O)c1ccccc1)C[C@@H]1CN2CCC[C@@H]2CO1', 'Clc1ccccc1N1NC[C@@H](Cc2cnn(-c3ccccc3)n2)C1', 'O=C([C@H](c1ccc(Cl)cc1)N1CC[NH2+]CC1)NCCOC1CC1', 'Clc1cccc(/N=C/c2ccc(-c3ccccc3)cc2)c1', 'O=[N+]([O-])c1ccn([C@H]2NC[C@@H](O)[C@@H]2O)c1', 'O=C(c1ccc(O)c(Cl)c1)N1CCC([NH+](Cc2ccco2)C2CC2)CC1', 'Clc1cccc(S[C@H]2CCC[C@@H](OCC(F)(F)F)C2)c1', 'Clc1nncc(NCCc2nc(C3CC3)no2)c1O', 'Oc1ccc([N+](=O)c2ccccc2)c(F)c1Oc1ccccc1F', 'Oc1ccc(/C=C/Nc2cccnc2)cc1', 'Clc1ccc(CN2CCO[C@@H](c3cccc(C(F)(F)F)c3)C2)cc1Cl', 'O=C(CCNc1ccccc1N1CCOCC1)NCc1cccs1', 'ClCc1ccc[nH]1', 'ClCc1ccc[nH]1', 'ClCc1ccc[nH]1', 'Clc1nnnn1CCCC[NH+]1CCC(Cc2ccccc2)CC1', 'ClC[C@H](c1nc2ccccc2s1)N1CCO[C@@H]2CCCC[C@H]21', 'Clc1ccc(-c2nn(-c3ccc(COCC(F)F)cc3)o2)cc1', 'Clc1cc(Cl)c(-n2cc(I)c3ccccc32)nc1', 'Oc1cc(F)nc(CN2CC[C@@H](N3CCO[C@@H](CO)C3)C2)c1', 'COc1nn(-c2c(Cl)cccc2Cl)c(-n2nc(C)cc2C)c1NC(=O)c1ccc(NC(C)=O)cc1', 'Clc1ccccc1-c1cscc1-c1ccc(Br)cc1', 'O=C(c1ccsc1)N(Cc1ccccn1)c1nc2ccccc2c(=O)[nH]1', 'Clc1ccccc1Cn1cnc2ccccc2c2nc(N3CCCC3)ncc21', 'Clc1ccc([C@@H](O)[C@H]([NH2+]Cc2ccccc2)C2CCC2)cc1', 'C=C[C@@H](CC(C)C)[NH2+]C1CCN(C(=O)Nc2ccccc2)CC1', 'COC(=O)C[C@H]1CCC[C@H]1NC(=O)NCc1cnn(C)c1', 'Clc1ccccc1Oc1cnc2cc(C(=O)NCc3ccccc3)[nH]c2c1', 'C[C@@H](Oc1ccccc1F)C(=O)Nc1cnn(CC(=O)NC2CC2)c1', 'Clc1ccc(Br)c(F)c1NCC1(CC2CC2)CCOCC1', 'O=C([C@H](Nc1ncccn1)c1ccccc1)[C@H]1CCCO1', 'O=C(/C(c1cccs1)CC1(C)CC1)Nc1ccccc1Br', 'Clc1ccc(Cc2noc(COc3ccccc3)n2)cc1', 'CCCCSC[C@@H](C)CNC(=O)c1cccc(Br)c1', 'Clc1ccccc1Nc1ncc2c(c1C(=O)[O-])C[NH+](Cc1cccnc1)C2', 'ClCc1ccnnc1N[C@H](c1ccccc1)c1ccccc1', 'Oc1ccc(F)cc1N1CCOC[C@H]1C(=O)N(Cc1ccccc1)C1CC1', 'Clc1nnc(CSc2ccc(N3CCCC3)cc2)nc1N1CCCC1', 'Clc1cccc2ccccc12', 'O=C(c1cnc(/N=C/CCO)cc1)OCc1nc(-c2ccncc2)no1', 'O=C(CSC(=S)N1CCc2ccccc21)NCCc1ccccc1', 'Oc1ccc(O[C@@H]2CC[NH2+]C[C@H]2Cc2ccccc2)cc1', 'N[C@@H]1[NH2+]CC[NH+]([C@H](Cc2ccccc2)c2ccccc2)C1', 'ClC(=S)Sc1ccc(N[C@H]2CCCC[C@H](C(=O)[O-])C2)cc1', 'Nc1ncc(CC[NH2+]C[C@H]2CCOC2)[nH+]c1N', 'Clc1cccc2ncncc1n2CCOc1ccccc1', 'O=C(COc1ccc(CN2CCOCC2)cc1)N[C@@H]1C[C@H]2CCCC[C@@H]2C1', 'C[C@@H](OCC1CC1)C(=O)N1CC[C@@H](Oc2ccccc2F)C1', 'Oc1cccc(OC2CCN(C/C=N\\CC[NH+]3CCC(Cc4ccccc4F)CC3)C2)c1', 'Clc1ccccc1OCCCc1nnc2n1CCC2', 'Clc1nnn(C(C)(C)Cc2ccc[nH]2)c1-c1ccccc1', 'Clc1nncc(OCCCO[C@H]2COc3ccccc3O2)c1[N+](=O)[O-]', 'COc1nc(N2CCCCC2)nc2c1cnn2Cc1ccccc1Cl', 'Clc1ccccc1CSCCOc1ccccc1F', 'Clc1ccccc1COCCSc1ccc(F)cc1', 'Clc1ccccc1CSCCc1ccccc1', 'O=C([C@H](Cl)Cc1ccccc1)c1ccccc1', 'O=[N+]([O-])c1ccncc1[C@@H]1CCCN(c2nccc(-c3ccc(F)cc3)n2)C1', 'OC[C@H]1[C@H]([NH2+]C2CCCCC2)CCCCC1(CCCC(F)(F)F)Nc1ccccc1', 'Clc1ccc(CNc2ccc(C(=O)NNC(c3ccccc3)nc2)C2CC2)cc1', 'Clc1ccc(-c2nnnn2CCNc2ccc(Cl)cc2)cc1', 'O=C(/C(c1cccs1)S(=O)(=O)c1ccccc1)[C@@H]1CC(=O)N(c2ccccc2)[C@H]1Cc1ccccc1', 'O=[N+]([O-])c1ccc([C@H](CCCO)N[C@H]2CC(c3ccccc3)=NO2)c2ccccc12', 'Clc1ccc(CCCC[NH2+][C@@H]2CCC[C@H](C(F)(F)F)C2)cc1', 'Clc1cc(COC[C@@H]2CCCO2)ccc1NCc1coc(-c2ccccc2)n1', 'C[C@@H](Cn1c(C[NH+](C)CCCN2CCOCC2)noc1=O)c1ccccc1', 'CN=Nc1nc(CNCc2ccccc2O[C@@H]2CCC[C@@H]2OCC(F)F)n[nH]1', 'O=C(CCc1nnc(-c2ccccc2)o1)NCCC[NH+]1CCCC1', 'OC[C@@]1(NCC2CC2)CCC[C@@H]1CCOc1cccnc1', 'ClC(/N=C1/CSC[C@@H]1N1CCCC1)c1ccc(SC(F)F)cc1', 'N=C1C=C(C(=O)N[C@H](c2ccc(OCN3CCOCC3)cc2)c2cccs2)c1', 'O=C(CCS(N)=C(Cl)Cc1ccccc1)NC[C@@H](O)c1ccc(F)cc1', 'O=C(CS(=O)(=O)[N-]c1ccc(OC(=O)c2cc(F)ccc2F)cc1)N1CC[C@H]([NH+]2CCCC2)C1', 'COC[C@H]1CCC[NH+](Cc2cccc(OC)c2OC)C1', 'Clc1ccc(Br)cc1[C@@H]1C[NH2+][C@@H](CO)[C@H]2NC(=S)N(Cc3ccccc3)C12', 'O=C([O-])c1ccc(-n2cccn2)cc1C(=O)NC[C@H]1Cc2ccccc2O1', 'Oc1cccc([C@H]2C[C@H]2C(=O)N2CCC(OCc3ccccc3)CC2)c1', 'Clc1ccccc1O/Cn1nc(-c2ccccc2F)nn1', 'Clc1ccccc1-c1cscc1C[NH2+][C@H]1CCc2ccccc21', 'Clc1ccccc1COCCSc1ccccc1', 'O=C(COc1cc(N2CCCCC2)nn1-c1ccccc1F)N1CCN(S(=O)(=O)c2cccs2)CC1', 'Clc1ccccc1[C@H](OCc1ccco1)C1CC1', 'Clc1ccccc1OCC[NH2+][C@@H]1CCc2ccccc21', 'Clc1cc(/N=C/c2nnc3ccccn23)cc(OCc2ccccc2)c1Cl', 'Clc1ccccc1Nc1ncnc(N2CCC(c3ccc(F)cc3)CC2)n1', 'Clc1cccc2[nH]nc(C[C@@H](C(=O)[O-])N3CCCC3c3ccccc3)c12', 'Clc1ccc(-c2nonc2CN2CCN(c3ccccc3Cl)CC2)cc1', 'OCc1n[nH]cc1OCC1CC[NH+](Cc2ccccc2)CC1', 'Clc1nncc(C[C@@H]2CCCCc3ccc(F)cc23)c1C1CC1', 'Oc1ccc(F)cc1S[C@H]1CCC[NH+](CC(F)F)C1', 'O=C(Cc1nc(-c2ccccc2)no1)NCc1cc(Cl)cc2ccccc12', 'Oc1cc(O)cc(C(=O)N2CCN(S(=O)(=O)Cc3ccc4c(c3)CCCC4)CC2)c1', 'OC[C@@]1(c2ccccc2)N[C@H](c2cccc(F)c2)C(=O)N1COc1ccccc1', 'OC[C@@H](C(=O)N1CCc2ccccc2C1)N1CCc2ccccc2C1', 'Clc1ccc([C@@H](OCc2ccccc2)c2ccccc2)cn1', 'Oc1ccc([C@H](Nc2cc(C(F)(F)F)c3ccccc3n2)c2ccncc2)cc1', 'CN(c1ccc(F)cc1)S(=O)(=O)c1ccc(NC(=O)[C@H]2CC(=O)N(c3ccccc3)C2)cc1', 'O=C(CN(c1cn[nH]c1)c1ccccc1)N1CCN(Cc2cccs2)CC1', 'Clc1cccc2c1-c1ccccc1C2', 'O=C(CNc1nn(CC[NH+]2CCCCCC2)c2c1CCCC2)Nc1ccccc1', 'Clc1ccc(Cc2noc(C[NH2+]C3CC3)c2)cc1', 'Clc1ccc(Cc2cc[nH]n2)c(C[NH2+]C2CC2)c1', 'Clc1cc(CNCc2ccc(C3CC3)cc2)n[nH]1', 'OCc1ncccc1CNC[C@H]1CC[NH+]=CC1', 'Clc1ccc(C[NH+](CCO)Cc2cccnc2)cc1', 'O=C(c1ccsc1)[C@@H]1C[C@@H](c2ccccc2)[NH2+]Cc2ccccc21', 'O=C(c1cnc(N/N=C/c2ccc(Cl)cc2)o1)N1CCc2ccccc21', 'O=[N+]([O-])c1ccn([C@H](CCO)c2ccc(F)cc2)n1', 'Clc1nnn(C[C@@H]2CCC[NH+](C3CCCC3)C2)c1Br', 'Clc1cc(CN(Cc2ccccc2)Cc2ccsc2)ccc1Cl', 'Clc1ccccc1Oc1cc(Cl)ccc1N1CCCCC1', 'OC[C@@]1(N2CCc3ccccc3C2)CCC[NH+]1C1CCCCC1', 'CNS(=O)(=O)CCNC(=O)N(C)Cc1ncc[nH]1', 'Clc1ccccc1Oc1cnc2ccc(C3(c4ccccc4)c(=O)n3[C@H]3CCOC3)nc2c1', 'CN(Cc1ccc2ccccc2c1)C(=O)CN1C(=O)N[C@](CNS(=O)(=O)c2ccccc2)C1', 'O=C(COc1cc(Nc2ccccc2)ccc1O)NNN1CCCCC1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1OC[C@@H]1CCCO1', 'O=C([O-])c1[nH]ncc1CNC(=O)C[C@@H]1CCCC[NH2+]1', 'CCC[C@]1(C)CCN(c2ccc([N+](=O)[O-])c([N+](=O)[O-])n2)C1', 'Clc1ccc(-c2cc(SCC(=O)NCC3CC3)ccc2Cl)cc1', 'O=[N+]([O-])c1cc(SC(CC2CC2)C(=O)[O-])ccc1Br', 'Oc1ccc(F)cc1OCCNC(=O)c1ccccc1Cn1cccn1', 'Clc1ccc(-c2csc3sc(=O)n(-c4ccccc4F)c23)cc1', 'OCc1n[nH]cc1Oc1ccc(C(=O)Nc2ccc(Cl)cc2)cc1', 'O=[N+]([O-])c1c(Cl)ncnc1N[C@@H](c1ccccc1)c1ccccc1', 'Clc1nnnc(O[C@@H](C(=O)c2ccccc2)c2ccccc2)c1N', 'O=C(CCCc1ccc2[nH]c(=O)[nH]c(=O)c2c1)Oc1ccccc1', 'Clc1ccc(COCc2nnc(-c3ccccc3OC(F)(F)F)o2)cc1', 'Clc1ccccc1[C@H]1[C@@H](C2CC2)Nc2ncnn21', 'NS(=O)(=O)CCNC(=O)[C@@H](C#N)c1ccc2c(c1)CCC2', 'Clc1ccc2ccccc2c1Oc1ccccc1C[NH2+][C@@H]1CCS(=O)(=O)C1', 'Clc1ccccc1-c1nn(C[NH+]2CCN(C[C@@H](O)c3ccccc3)[C@@H](O)C1)C2', 'CC[C@@H]1C[NH+](Cc2ccccc2)C[C@@H](C)C1', 'Clc1ccc(-c2nnnn2C[C@H](O)c2ccccc2)cc1', 'Oc1ccc(Oc2ccc(-c3noc4cc(F)ccc34)nc2)cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])c(Br)cc(OCc2cccs2)c1C#CCOc1ccccc1', 'Clc1nncn1c1cnn(C2CCC(Oc3ccc(Cl)cc3)CC2)c1', 'Clc1ccc(-c2nonc2N2CCC(Oc3ccc(F)cc3)CC2)cc1', 'Clc1ccccc1C1CC1', 'O=C(/N=c1\\[nH]c(S)nc2ccccc12)N1C[C@@H](O)[C@H]2C[C@@H]2[C@@H]2CCCC[C@H]21', 'Clc1ccc(Br)cc1NCCc1c[nH]c2ccccc12', 'O=C(c1cccnc1)N(CC1CCN(C(=O)c2cnco2)CC1)C[C@H]1CCCO1', 'Clc1nncc(-n2nc(N3CCc4ccccc43)C2)c1C1CC1', 'O=[N+]([O-])c1ccn(C(=O)CBr)c1', 'O=C(C1CC1)N(CCO)Cc1ccccc1Cl', 'Clc1ccccc1[C@H]([NH3+])C[NH2+]C[C@@H]1CCCO1', 'CCCC1CC[NH+]([C@H](C)c2nnnn2CC)CC1', 'Clc1ccccc1Cc1ccc(O[C@@H]2CC[NH2+]C2)c(Br)c1', 'Clc1ccccc1O/C=C[C@@H](O)COC1CCCC1', 'Clc1ccccc1COCCOC[C@@H]1CCCO1', 'COCC[C@@]1(Cc2ccc(F)cc2)NCCn2cccc21', 'Oc1ccccc1/N=N/n1c2ccccc2c2ncnn21', 'O=[N+]([O-])c1ccncc1NCCC(=O)N1CCC[C@@H](c2nccs2)C1', 'Clc1ccc(-c2n[nH]c(C[NH+]3CCC(COc4ccc(F)c(F)c4)CC3)n2)cc1', 'Clc1ccccc1Oc1cnc(C2CC2)n1C1CC1', 'Clc1ccc(Br)cc1-n1cc[nH+]c1CN1C[C@@H]2CC[C@H](C1)C2', 'O=C(C[NH+]1CCO[C@]2(CCCN(c3ncccn3)C2)C1)NC1CCCCC1', 'Clc1ccc(Br)c(OC[C@H]2CC[NH+](CC(F)(F)F)C2)n1', 'Clc1ccccc1OCc1cc(OCc2ccc(F)cc2)nc2ccccc12', 'O=C(CCN1CCOCC1)N(c1ccccc1F)[C@@H]1CCS(=O)(=O)C1', 'O=[N+]([O-])c1cc(SC[C@H]2CC=CCC2)ccc1O', 'Clc1ccccc1Cc1noc(CCN2CCOCC2)n1', 'Clc1ccc([C@@H]([O-])CCc2nc(C3CCCCC3)no2)cc1', 'Clc1ccccc1C1(C[NH2+]Cc2ccccn2)CC1', 'Clc1cccc2[nH]nnc1c1nc2cc(Cl)ccc1OCC1CC1', 'O=C(COc1cc(Cl)c2c(c1)OCCO2)Oc1ccc(-c2ccccc2)cc1', 'Clc1ccc(-c2ccccc2C2(c3ccc(Cl)cc3)CCOCC2)cc1', 'O=C(CCc1ccccc1)N1CCC(NC(=O)Nc2ccnc(F)c2)CC1', 'O=C(/N=C/CC1CC[NH+](Cc2ccccc2)CC1)C1CCCC1', 'ClCc1cccc(F)c1OC[C@@H]1CCCCC[C@H]1CO', 'O=C(/N=C\\NCSc1cccc(Cl)c1)N1CCOC[C@@H]1C1CC1', 'Clc1ccc([C@@H]([O-])C[NH+](Cc2ccco2)C2CC2)cc1', 'CNC(=O)c1ccc(OCCNC(=O)c2cnn(CC)c2)cc1', 'ClCc1nncn1CC(=O)N[C@H]1CCN([C@@H](c2ccccc2)c2ccccc2)C1', 'O=C(c1cc([N+](=O)[O-])[nH]n1)N(Cc1ccc(Cl)cn1)[C@H]1CCS(=O)(=O)C1', 'Clc1ccc(Cc2c[nH]nc2C[NH+]2CCCC[C@@H]2c2cccnc2)cc1', 'O=C(C1=CN(c2cccc(Oc3ccccc3)c2)CCC1)N1CCc2nc[nH]c2C1', 'Clc1ccc(Br)c(OCc2csc(-c3cccn3CCC(F)(F)F)n2)c1', 'O=[N+]([O-])c1cccc2c1NC(=O)C[C@H]2c1ccccc1NCC1(Cl)CCCC1', 'Oc1ccc(F)cc1SCC(=O)Nc1ccc2c(c1)COC2', 'Oc1ccc(/C=N\\CC[NH+]2CCCCC2)cc1', 'Nc1[nH]nc(CCC(=O)N[C@@H]2CCCC[C@@H]2Cc2ccccc2)n1', 'Cc1ccsc1CN1CCO[C@H](c2cccc([N+](=O)[O-])c2)C1', 'Clc1ccccc1OC1(OCc2ccccc2)CCCC1', 'O=C(c1cnn(C[C@@H]2CCCO2)c1)N1CCC(c2nc3c(F)cccc3s2)CC1', 'OC[C@@]1([NH3+])CCC[C@H](Cc2ccncc2)C1', 'Clc1ccc(C2SCCCCCC2)cc1F', 'O=[N+]([O-])c1c(F)c(Cl)cnc1NCc1ccc(N2CCOCC2)cc1', 'O=C(/C=C/OCCOc1ccc(Oc2ccccc2)cc1)Nc1ccc(Cl)cc1', 'Clc1ccccc1Oc1cnc(COc2ccccc2)[nH]1', 'ClC(=S)c1ccc(N[C@@H]2CCc3c(Cl)cccc32)cc1', 'O=C(c1ccco1)N1CC(Cn2cc[nH+]c2)c2ccccc21', 'Clc1ccccc1Oc1cnc2cc(-c3cccc(C4O[C@H](CO)C4)c3)[nH]c2n1', 'O=[N+]([O-])c1c(-c2cccnc2)ncc(C(=O)Nc2ccccc2F)c1C1CC1', 'O=C(c1cccn1c1ccc(Cl)cc1)N1CCOC[C@@H]1CO', 'Clc1ccc(C[NH2+][C@@H](CO)c2ccco2)cc1', 'Oc1cc(CC[NH+]2CCC[C@H]2c2ccco2)ccc1N1CCOCC1', 'Clc1ccccc1CNC[C@@H](O)COc1ccccc1', 'Clc1ccc([C@@H](Cl)Cc2nccn2Cc2ccccc2)cc1', 'Oc1ccc([C@H]2[NH2+]C[C@H](Nc3ccc(S(=O)(=O)N4CCOCC4)cc3)CSC2=O)cc1F', 'Clc1ccc(C(F)(F)F)nc1OCCCOc1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnnc1CSC1CCC1', 'O=C(c1cnc(N[C@H]2CCOC2)cn1)Nc1ccc(Cl)c2ccccc12', 'Clc1cccc(Br)c1N[C@@H](c1ccc(F)cc1)N1CCCC1', 'O=[N+]([O-])c1ccc2ncn(CC(=O)N3CCC([C@H]4CCOC4)CC3)c2c1', 'Oc1cc(O[C@@H](C[NH+]2CCCCC2)c2ccccc2)ccc1OC[C@@H]1CCCO1', 'Clc1cccc(Br)c1SCC(F)F', 'Clc1cccc(F)c1N1CCC(N/C=C2\\C3=C(CCO3)CC2)C1', 'O=C([C@H](Cl)Cc1ccccn1)[C@@H]1CCC[C@@H](O)C1', 'Clc1cc(/C=Cc2cccc[n+]2[O-])cn1C1CC1', 'OC[C@@H](C[C@@H]1CCCO1)N(CC(=O)Nc1ccccc1)c1ccccn1', 'O=[N+]([O-])c1cc(Cl)n(-c2ccccc2)c1NC[C@@H]1CCCO1', 'Clc1ccccc1Cn1cccc(Br)c1=O', 'Clc1ccccc1OCc1ccc(C(=O)NCc2ccco2)cc1', 'Clc1ccc(CN[C@@H](C(=O)N2CCCC2)c2ccccc2)nc1', 'Clc1ccc(-c2nnnn2CCC[NH+]2CCCCCC2)nc1', 'Clc1ccc(-c2nnnn2/C=C/CC[NH+]2CCCCCC2)nc1', 'O=C(COc1cc(C2(O)CCCCCC2)cc(Br)c1)NCC1CCCCC1', 'Clc1ccc(-c2n[nH]cc2C[NH+]2CCc3[nH]c4ccccc4c3C2)cc1', 'N=Cc1ncc(CN(C)C)cc1S(=O)(=O)Nc1ccc(Oc2ccc(C)cc2)cc1', 'O=C(Cc1cc(N2CCCC2)ncn1)N1CCCCCC1', 'CNC(=O)[C@@H](NC(=O)c1cnn(-c2ccccc2)c1)c1nc2ccccc2cc1Cl', 'Cc1sncc1C(=O)Nc1nc(-c2ncc(-c3ccc(F)cc3)nc2)no1', 'Clc1ccccc1Cc1cn2ccccc2n1', 'Clc1ccc(-c2nonc2CC2CCCC2)cc1', 'C[C@@H](C(=O)Nc1ccccc1F)[C@H](C)c1ccccc1', 'O=[N+]([O-])c1c(F)cccc1N1CC[C@@H](C[NH+](Cc2ccccc2)C2CC2)C1', 'O=C(Cc1c[nH]cc1Cl)NCc1nc(C2CCCCC2)no1', 'Clc1ccc(-c2nonc2NCCN2CCOCC2)cn1', 'O=C(CN(c1ccccc1)S(=O)(=O)c1ccc(F)cc1)NC[C@@]1(O)CCc2ccccc21', 'ClCc1cn[nH]c1NC[C@H]1CNc2ccccc2C1', 'Oc1ccc(O[C@@H](CO)C(=O)NCc2ccccc2F)cc1', 'Oc1cccc(NC2CCN(C(=O)N[C@@H]3CCN(C(=O)c4ccncc4)CC3)C2)c1', 'O=[N+]([O-])c1cc(N2CC[NH+](CC3=c4ccccc4=[NH+]C3)CC2)ncc1Cl', 'Clc1ccccc1[C@H]([NH2+][C@H]1CCc2cccnc21)c1ccccc1', 'O=[N+]([O-])c1ccc[nH+]c1Nc1ccc(/C=C/C(=O)[O-])cc1', 'Clc1ccc(-c2nonc2NC[C@H]2CCCO2)cc1', 'Clc1ccccc1NCCc1ccc2c(c1)CCCN2Cc1ccccc1', 'Clc1ccc(CNSc2ncnc3c2C[NH+](CC(F)(F)F)C3)cc1', 'Clc1ccc([C@@H](NC[C@@H]2CCCCO2)c2ccc(Cl)cc2)cc1', 'Oc1ccc(O[C@H](C[NH+]2CCCC2)c2ccccc2)cc1', 'Clc1ccc(CN2N=C3SCC=C3SC2)cc1N1CCCC1', 'O=C(CCC(=O)N1CC[C@@H](CCCO)C1)Nc1cccc(OCC(F)(F)F)c1', 'Oc1ccc(Oc2ccc([C@H]3CCCN(c4ncccn4)C3)nn2)cc1', 'N#Cc1c(SCCC(=O)Nc2ccc(Br)cc2)n[nH]c1NC(C)C', 'Clc1ccsc1/CC1(C#N)CCN([C@@H]2[C@H](CN3CCN(c4ccccc4)CC3)C2)C1', 'Oc1ccccc1[C@@H]1COc2ccccc2N1CN1CCOCC1', 'O=C(c1c[nH]c(C(=O)N2CCC[C@H](c3nc4ccccc4[nH]3)C2)n1)N1CCCC1', 'O=[N+]([O-])c1c(F)ccc(CNCc2nc(-c3ccncc3)no2)c1Cl', 'Clc1ccccc1Oc1cc(NCCc2nc3ccccc3o2)ccn1', 'Oc1cc(OC(=O)c2ccc(Cl)cc2F)ccc1N[C@H](c1ccccn1)C1CCCC1', 'Oc1ccccc1[C@@H]1CCCN1C(=O)NCCCCn1cncn1', 'O=[N+]([O-])c1ccncc1COC[C@H]1CCCC[C@@H]1n1cc[nH+]c1', 'Nc1[nH]nc(N(CCc2ccccc2)C2CCCCC2)c1[N+](=O)[O-]', 'NS(=O)(=O)N1Cc2ccccc2S(=O)(=O)[C@@H]1Cc1ccccc1-n1cccc1', 'O[C@@H](Sc1cc2c(cc1[N+](=O)[O-])CC[NH2+]C2)C(=O)[O-]', 'Clc1ccccc1OCCO[C@H]1C[C@@H]1C1CCCC1', 'Clc1ccccc1OCCN1CC[NH+](CC2CC2)CC1', 'Clc1ccc(Br)cc1O[C@H](c1ccccc1)[C@@H]1CCCO1', 'O=C(Cc1cn(-c2ccccc2)nn1)NCc1cccc(CN2C(=O)[C@H]3CCCC[C@H]3C2=O)c1', 'Clc1cccc(SC[C@H]2CCCC[C@@H]2[NH2+][C@@H]2CCOC2)c1', 'CN[C@H]1[C@H](OCC(C)C)CCCc2cccc(Br)c21', 'Clc1ccc([C@@H]([O-])c2ccc(OCC[NH+]3CCC(CO)CC3)cc2)cc1', 'O=C(c1cnn2c1CCCC2)N1CCO[C@H](c2ccc([N+](=O)[O-])cc2)C1', 'Clc1ccc(-c2n[nH]c(CO[C@@H]3CC[C@@H]4CCCC[C@@H]4C3)c2[N+](=O)[O-])cc1Cl', 'Clc1nnnc(NNc2ccc(S(=O)(=O)N3CCCCC3)cc2)c1Cl', 'COc1ncc(F)c(C[NH2+]Cc2ccc(SC)cc2)c1C#N', 'Clc1ccc(COc2ncc(C[NH+]3CCCC3)[nH]2)cc1', 'OC[C@H](C[C@@H]1CCC[NH2+]1)c1nc(-c2ccccc2)no1', 'Clc1ccccc1CCCn1c(C[C@@H]2CCS(=O)(=O)C2)nc2ccccc21', 'ClC[C@H]1CCc2ccccc21', 'Clc1ccc(Br)cc1N1CCN(CC2[NH+]=c3ccccc3=[NH+]2)CC1', 'O=[N+]([O-])c1cc(CNc2ccc(Oc3ccccc3)nn2)cc2c1OCCO2', 'O=C(c1cncc([N+](=O)[O-])c1)N1[C@@H](CO)CC1(c1ccccc1)c1ccccc1', 'Clc1ccc(-c2nnnn2C[C@]2(O)CCOC2)cc1', 'COc1nn[nH]c1CC(=O)/C=C/c1cccc([N+](=O)[O-])c1', 'C[C@@H]([C@@H](O)[C@@](C)(C(N)=O)[NH2+]C1CC1)c1ccccc1', 'Clc1ccccc1Cc1nc(Cn2ccccc2=O)no1', 'Clc1ccc(Cc2noc(Cc3ccncc3)n2)cc1', 'Clc1cccc(Br)c1S[C@H]1CCOC[C@@H]1C(=O)[O-]', 'N[C@@H]1[C@H](C(=O)OCCSc2ccccc2)Cc2ccccc21', 'CC[C@@H](COC)[NH+]1CC2(CC[NH+](Cc3ccccc3)CC2)C1', 'Oc1ccc([N+](=O)Cc2n[nH]c(Cn3c(=O)[nH]c(=O)n3C)c2C(F)(F)F)cc1', 'Clc1cccc(F)c1[C@@H]1CCCN1Cc1ccccc1F', 'Oc1ccccc1[C@H](NNC(=O)c1ccc2nccnc2c1)c1ccccc1CO', 'CCCC[C@@H](CC)CNC(=O)N[C@H]1c2ccccc2C[C@H]1O', 'Oc1cc(F)ncc1-c1ncc(Cl)cc1O[C@H](CO)C(=O)[O-]', 'Oc1ccccc1[C@H](N[C@H]1C[C@@H](O)[C@@H](O)c2ccccc21)c1nnc2cccnn12', 'Clc1ccccc1Oc1cnc(NN2CCOCC2)nc1', 'Clc1ccc(-c2cccs2)c(NC[C@H]2[C@@H]3CC[C@@H]2C3)c1', 'N=Cc1ccc([C@@H](C)NC(=O)C2CC2)cc1', 'Oc1ccccc1CC(=O)N1CCOCCN1c1ccccc1', 'O[C@@H](Sc1ccccc1)C(=O)N(Cc1ccc(Cl)cc1)C1CC1', 'Clc1ccc(COc2nc(C3CC3)n(-c3ccccc3)n2)cc1', 'Clc1ccc([C@@H](CC[C@H]2CCOC2)NN=C2CCCOC2)cc1', 'Oc1ccccc1-c1ccccc1COc1ccccc1Cl', 'Oc1cc(O)cc(/C=C(/C(=O)NCCc2ccccc2Cl)C(F)(F)F)c1', 'Clc1ccccc1Oc1cnc(CNc2ccccc2)o1', 'Clc1ccccc1Nc1nn[n-]c1COc1ccccc1', 'O=C(c1ccsc1)N(C1CC1)[C@@H](CCCn1cncn1)c1ccc(F)cc1', 'Oc1ccc(F)c(SCC[NH2+]Cc2ccc(Cl)cc2)c1', 'O=[N+]([O-])c1c([O-])cnn1-c1cccc2c1CCN(Cn1cncc1Cl)C2', 'Oc1ccc(F)cc1CSC1=N[C@@H](c2ccc(F)c(F)c2)N1C1CCC1', 'Clc1ccccc1CCCCC(=O)Nc1ccc(Cl)cc1F', 'Clc1cc(Br)ccc1OCC(=O)N[C@H]1c2ccccc2C1', 'Clc1cccc(S[C@@H](C(=O)CCC2CC[NH+](C3CCOCC3)CC2)c2ccccc2)c1', 'ClCc1noc([C@@H]2CCCCC[C@@H]2[NH2+][C@@H]2CCCC[C@@H]2O)n1', 'O=[N+]([O-])c1c(Br)sc2c1[C@H]1CC[C@@H]2C[C@@H](O)[C@@H]1N1c2ccccc2C1=O', 'Oc1cccc(Br)c1OCc1c[nH]c2nccc(Oc3ccccc3C(F)(F)F)c12', 'COCc1nc(C)ccc1C(=O)O[C@@H](CC)c1ccccc1', 'NS(=O)(=O)N1CCC(CNC(=O)N2CC[C@@H](C(=O)Nc3ccccc3)C2)C1', 'O=C(C1CC1)[NH2+]CCCc1ccccc1F', 'ClC([O-])=CCC[C@H](c1ccccc1)N1CCCCCC1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccn1Cc1ccc(Cl)c2ccccc12', 'CN(CCCNC(=O)[C@@H]1COc2ccccc21)C1CC1', 'O=C(CSC(c1ccccc1)c1ccccc1)N1CCc2ccccc21', 'OCS[C@@H](C1CCCCC1)C(c1ccccc1)c1ccccc1', 'O=C(c1cnn(-c2ccccc2)c1)N1CC2(CCCCC2)Oc2ccccc21', 'Clc1ccccc1Oc1cc(OC2CCCCC2)c[nH]c1=O', 'C[C@@H](c1cnn(C)c1)c1ccccc1C[NH+](C)Cc1cnc[nH]1', 'O=[N+]([O-])c1c(OCO)oc2c1COCCN(c1ccccc1)C2', 'OC[C@@H]1c2ccc(F)cc2CCN1C(=O)CCCCn1cccn1', 'O=C(c1ccsc1)c1nc2nc(NC(=O)/C=C\\c3ccccc3)noc2c1Br', 'Cc1cn(CC(=O)N(CCN2CCOCC2)Cc2ccccc2F)nc1[O-]', 'O=C(Cc1nc(=O)n(-c2ccc(Cl)cc2)[nH]1)NCc1ccc(F)cc1', 'O=[N+]([O-])c1cccc2nnc(C[NH+]3CCC[C@@H](CCc4cccnc4)C3)nc12', 'Clc1ccsc1N[C@@H]1CCCC[C@H]1COc1ccccc1F', 'Clc1nncnc1N1CCCC[C@@H]1Cn1cncn1', 'Oc1ccc(O[C@@H]([NH3+])C[C@@H]2OCc3ccccc32)cc1', 'N[C@@H](NC1CCN(c2ccccc2)CC1)c1ccccc1', 'C[C@@H]([C@@H](O)[C@@H](c1ccncc1)n1cccn1)N1CC[NH+](C)CC1', 'ClCc1nonc1N/N=C/c1ccc(-c2ccccc2)cc1', 'NS(=O)(=O)C[C@H](Cc1c[nH]c2ccccc12)NC(=O)c1ccccc1[N+](=O)[O-]', 'Clc1ccc(-c2cccnc2)nc1C[C@@H](O)COc1ccccc1Br', 'O=[N+]([O-])c1c(F)cc(N/N=C/c2sc3ccccc3c2CO)cc1OCc1ccccc1', 'CC[S@](=O)CC(=O)N1CCN(C(=O)Nc2ccccc2-n2cncn2)CC1', 'Oc1cc(F)cc([C@@H]2CSc3ccccc32)c1', 'Clc1ccccc1-c1nncn1[C@@H]1CC[C@@]2(CCCN(Cc3ccc(F)cc3)C2)C1', 'O=[N+]([O-])c1ccn([C@H](OC[C@H](O)c2cccc(Cl)c2)[NH+]2CCCC2)c1', 'Clc1ccc(C(CCn2cccn2)C(F)(F)F)cc1', 'O=C(COc1cc(C2CCN(c3ncccn3)CC2)on1)NCc1ccccc1', 'Oc1ccccc1[C@H](NCC1(CO)COC1)c1ccccc1', 'Oc1cccc(SC2[C@H]3CCCC[C@H]23)n1', 'Oc1ccc(/C=N/NNc2ccc[n+]([O-])c2)cc1', 'O=C([O-])/C(=C\\C(=O)Nc1ccccc1OC(F)(F)F)Nc1nc(C2CC2)n(-c2ccccc2)n1', 'Clc1ccc(CC[NH2+]Cc2ccccc2)cc1', 'Clc1cc(F)ccc1NCC(=O)N1CCC2(CCSCC2)C1', 'Clc1ccc(-c2cccnc2N2[C@@H](O)CCC[C@H]2OCc2ccccc2)cc1', 'O=C(CNc1nn(C2CC2)c(=O)c2ccccc12)NCc1csc(Nc2ncccn2)n1', 'OC[C@@H](SCc1cccc(C(F)(F)F)c1)N1CCN(C2CCN(C(=O)c3ccccc3)CC2)C1', 'CCn1cc([C@@H]2C[C@@H]2C(=O)O[C@H](C)C(=O)NCC[C@@H]2CCCO2)s1', 'Oc1ccc([N+](=O)[O-])c(S[C@@H]2CC[NH+](Cc3ccccc3)C2)c1', 'CN(c1ccn(C)c1)CNC(=O)CCc1c(O)cccc1F', 'Clc1ccc(CCC[NH2+]C[C@H]2CCC[C@@H]2c2ccccc2F)c(F)c1', 'Clc1ccccc1C(=O)N(Cc1ccncc1)Cc1ccccc1C(F)(F)F', 'Oc1cc(CCN[C@@H](c2ccc(F)cc2)c2ccc3c(c2)NC(=O)CCC3)ccn1', 'Nc1cnn(CCn2cc(Cl)c(=O)n(-c3ccccc3)c2=O)c1O', 'Clc1ccc(Br)c(NCc2cccnc2C2CC2)c1O', 'Clc1ccccc1-c1csc(CC[NH2+]C2CCC3(CC3)C2)n1', 'N=CC[C@H]1C(=O)NC(=O)C[C@@]1(O)CSc1ccccc1', 'Clc1cccc(F)c1-c1[nH+]ccn1CCC[NH+]1CCCC1', 'O=C(Cc1conc1C1CC1)NC[C@@H]1CC[C@@H]([NH2+]C(C)C)C1', 'O=C(COc1cc(F)ccc1Br)N[C@@H]1CCSS1', 'Clc1ccccc1Oc1cnc2scc(-c3ccccc3F)c2c1', 'Clc1ccc(Cc2noc(C3CC3)c2[N+](=O)[O-])cc1', 'Oc1cccc(C[C@@H]2CCCN2C(=O)c2ccnc(NC3CCCCC3)c2)c1', 'Oc1cc(-n2cccc2)cc(Cn2ccc3ccc(Cl)cc32)c1', 'Clc1ccccc1O/C=C(O[C@H]1CCSCC1)c1ccccc1', 'Clc1ccc(Br)cc1OCCn1ccccc1=O', 'ClC[C@@H]1OCCc2c1[nH]c1ccccc21', 'Clc1ccc([C@@H](Br)C[C@@H]2CCOC3(CCCC3)C2)cc1', 'OCS[C@H](CCc1ccccc1)c1ccc2c(c1)OCO2', 'Clc1nnnn1/N=C/c1ccc([N+]2=C(CCCc3ccccc3)C(=O)NC2=S)cc1', 'Oc1ccccc1/C(Cl)Oc1cccc(C(F)(F)F)c1F', 'Clc1ccccc1Cn1nnc(-c2ccc(Cl)cc2)n1', 'Clc1nncnc1[N-]c1ccc(OCc2ccccn2)cc1', 'Clc1ccccc1C(=O)OCc1nc(Nc2ccccc2)no1', 'Clc1nnnn1CCOC1CC1', 'Clc1nnn(C[C@@H](CO)C2CC2)c1Cl', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(SCCCO)n1', 'N=CCn1ncnc1SCC(N)=O', 'Clc1cccc([S@@](=O)CCCC[NH+](C2CC2)[C@@H]2CCCO2)c1', 'O=C(Cc1cn2cccnc2n1)NCc1ccnc2ccccc12', 'Clc1ccc([C@@H](NS(=O)(=O)c2ccc(Cl)cc2)N2CCCC2)cc1', 'OC[C@@]1(NC2CC2)COc2ccccc21', 'O=C(C1=N[C@@H](CC(F)F)N1c1ccc(Cl)cc1)N[C@@H]1CCS(=O)(=O)C1', 'Clc1nncc(Sc2ccc(Cl)cc2)c1NC1CCCCC1', 'O=[N+]([O-])c1c(F)cc2n1[C@@H](c1ccccc2F)c(C2CC1)CN2/N=C=C1/N=NCCS1', 'CNC(=O)c1cc[n+](CCC(=O)N[C@@H](C)CCOC)cc1', 'OCS/C=C/c1cnc(C2CCCCC2)nc1Cl', 'O=C(c1ccsc1C1CC1)N1CCCCC[C@@H]1c1ccc(F)cc1', 'Nc1[nH]ncc1C[NH+](Cc1ccsc1)C[C@@H]1CCC[NH+](C2CCCCC2)C1', 'Clc1cc(/C=C2\\SC(=S)N(C3CC3)C2=O)cs1', 'O=C(/N=c1\\ocnn1Cc1ccccc1)c1cnn(C2CC2)c1', 'O=C(c1cn(C[C@H]2CCCO2)cc1)N1CCC[C@@H]1CF', 'O=C(C1=CC[NH+](CCCN2CCOCC2)CC1)Nc1cccnc1', 'Clc1ccc(C2=C[C@H](S(=O)(=O)Cc3ccc(Br)cc3F)N2[N+](=O)[O-])cc1', 'Clc1ccccc1CC(C[C@@H](O)CO)NNC(=O)c1ccsc1', 'Clc1cc(/C=N/N2CCN(C3CCCCC3)CC2)ccc1Br', 'Clc1ccc([C@@H](OCC(F)F)c2cccnc2)C[C@@H]1CC(=O)Nc1ccc(F)cc1', 'Clc1ccccc1C[NH+]1CCN([C@@H]2CCO[C@]3(CCCO3)C2)CC1', 'Clc1ccccc1COCCCn1cccn1', 'Clc1ccccc1CCCNC(=O)N[C@@H](C1CCCCC1)c1ccc(F)cc1', 'O=[N+]([O-])c1ccc(F)cc1CNC(=O)N1[C@@H](C(=O)[O-])CCN1c1ccccc1', 'C[C@@H]([NH2+]C[C@@H]1CCCC[C@H]1CCOc1ccccc1)c1ccc(F)c(F)c1', 'O=[N+]([O-])c1ccc(O)c(NCc2cnn(C3CCCCC3)n2)n1', 'CN(CC1CC[NH+](Cc2ccon2)CC1)C(=O)c1c[nH]cn1', 'CNS(=O)(=O)C[C@H](C)[NH2+]Cc1ccc(Cl)c2ccc(F)cc12', 'OC[C@@H](Cn1cc(Cl)cn1)NCc1ccccc1', 'O=[N+]([O-])c1ccncc1[C@@H](OCC(=O)N1CCCC1)c1ccccc1', 'Clc1ccccc1CO[C@H](c1ccco1)c1ccccc1', 'Clc1cc(/C=N/OCN(Cc2ccco2)CC2CC2)ccn1', 'Nc1[nH]nc2nc(CN3C[C@H](CCO)C[C@@H]3c3ccccn3)ccc12', 'Clc1ccc(-c2cccnc2)cc1', 'CCN1C(=O)c2ccccc2[C@@H]1C(=O)Nc1ccc(Br)cc1', 'Clc1nnnn1-c1c(-c2ccc(F)cc2)[nH]c2ccccc21', 'COC[C@H](O)C[C@@H](C)C(=O)N1CCN(c2ccc(F)cc2)CC1', 'O=C(C1CCN(c2ccc(F)cc2F)CC1)C1CCC(c2cn[nH]c2)CC1', 'Clc1cccc([C@@H]2[NH2+]CC[C@@H]2[C@@H]2CCCN2C(=O)C2CC2)c1', 'Clc1ccc(CCO)c(NCc2ccccc2Br)c1', 'Clc1ccc(-c2csc3s/Cc4ccccc4c23)cc1', 'O=C(Cc1cnn(-c2ccc3c(c2)OCO3)c1C#N)Nc1ccc2c(c1)OCO2', 'Nc1[nH]ncc1CCC(=O)Nc1ccn(-c2ccccc2)n1', 'C[C@@H](Oc1ccncc1)C(=O)Nc1nc(-c2ccccc2)ccc1F', 'N=CC1=C[C@@H](Sc2nnnn2C2CCC(C(N)=O)CC2)C(=O)N1c1ccccc1', 'Clc1ccc([C@@H](CC(=O)NCCN2CCCC2)C(F)(F)F)cc1', 'COc1nc(C[NH+]2CCN([C@@H](C)CC(F)(F)F)CC2)c2ccccc2n1', 'Clc1ccccc1O/N=C/c1ccc(-c2ccccc2)o1', 'Clc1ccc(C[NH+](Cc2ccc(F)cc2)[C@H]2CCOC2)cn1', 'O=C(/C(Cc1cccnc1)Nc1nc(-c2cccs2)cs1)[C@@H]1COc2ccccc2N1', 'O=C(Cc1n[nH]c(=O)[nH]1)NC1(c2ccccc2)CC1', 'Clc1ccc(-c2csc3sc(CCNC(=O)Cc4ccccc4)nc23)cc1', 'Oc1cc(O)cc(N2CCN(C(=O)c3ccc(COc4ccccn4)cc3)CC2)c1', 'OC[C@@H](C(=O)NCCCn1cccn1)Nc1ccccc1F', 'Clc1ccccc1OC1([C@@H](O)c2ccc(F)cc2)CC1', 'Clc1cccc([S@@]C2SCCCC2)c1', 'Clc1cccc(SC2CCC2)c1', 'ClC[C@H]1CCS[C@@H]1c1cccc(Cl)c1', 'O=C(c1cn(Cc2cccnc2)c2c1CCCC2)N1CCCCCCC1', 'C[C@@H]([C@@H](O)c1ccccc1)N(C)CC(=O)Nc1ccc(F)cc1Cl', 'Oc1cc(Br)c([C@H]2NN3[C@H](CC2)OC3C(=O)C2=CCCCC2)cc1Cl', 'O=C(Cc1c(N)nc(-c2ccccc2)o1)c1ccccc1', 'COC[C@@H](O)CN1CCN(C(=O)NCCc2ccc(F)cc2)CC1', 'O=[N+]([O-])c1ccncc1CC[NH+]1CCCCC1', 'O=C(c1ccsc1I)N(CC1CC1)C1CC1', 'COc1nn[nH]c1/C=C/C(=O)Nc1ccc(N2CCCCCC2)cc1', 'OCc1nc(C(=O)NCc2[nH]ccc2Cl)n[nH]1', 'Oc1ccccc1OC1CN(C(=O)c2cc(C3CC[NH+](C)C/C=C[C@H]3CO)[nH]n2)CC1', 'O=C(CS(=O)(=O)CCN1CCC([NH+](CC)C2CC2)CC1)c1ccc(F)cc1F', 'CNS(=O)(=O)Cc1ccc(C(=O)N2CCC[C@@H](Nc3cc(C)ccc3C)C2)cc1', 'O=[N+]([O-])c1ccnc2c1[C@H](NCc1ccco1)C2', 'Clc1ccc(-c2nnnn2CCSCc2ccc(Cl)o2)cc1', 'O=C(C1CCCCC1)N(Cc1ccccn1)c1ccccc1', 'Clc1ccc(-c2n[nH]cc2COc2ccc(F)cc2)cc1', 'Oc1ccccc1[C@@H](CO)Nc1ccccc1OCc1cccs1', 'N=Cc1cccn1CC(=O)N1C[C@H](C)OCC1(C)C', 'Clc1ccccc1Oc1cc(C[NH2+][C@@H]2CCOC3(CCCCC3)C2)ncc1SC', 'Clc1cc(Br)c2nc(OC3CCCC3)[nH]c2c1', 'O=C(c1cn(C2CC[NH2+]CC2)nn1)N1CCN(c2ccc(F)cc2)CC1', 'Clc1ccc(COc2nccnc2Cc2ccccc2)cc1', 'Nc1nonc1CN1CCN(C(=O)/C=C/C2CC2)CC1', 'Clc1ccccc1OCCCCNc1ncnc2c1CCC2', 'O=C(C1=CCCCN1)N1CCN(c2ccc([N+](=O)[O-])c(NN)n2)CC1', 'Oc1cc(O[C@@H]2OC(=O)c3ccccc3C2)ccc1OCc1ccccc1', 'Clc1ccccc1Oc1cnc(CCn2ccccc2=O)cc1', 'Clc1ccccc1NS[C@H]1CCOC2(CCCC2)C1', 'Oc1ccccc1OS[C@H](c1ccccc1)N1CC[C@@H]([N+](=O)[O-])C1', 'O=C(COc1ccc(S(=O)(=O)N2CCCCCC2)cc1)NNC(=O)CCl', 'Clc1ccc(Br)cc1N1CCC[C@H]1C[NH+]1CCCC1', 'Clc1ccc(Br)cc1C[C@H]1CCC[NH+](CC2CC2)C1', 'Clc1ccc(-c2nnnn2Cc2ccccc2Cl)cc1', 'O=C(C1CC1)N/C=C/c1ccnc2ccccc12', 'Clc1ccccc1O/Cn1nnnc1Cc1cccs1', 'Clc1nnn(C[C@@H]2CCCC[C@@H]2Cc2ccc(F)cc2)n1', 'Clc1ccc([C@@H](Br)Cc2ccccc2)cc1', 'Oc1cc(-n2nnc(/C=C/c3ccccc3)c2N2CCCCC2)cc(Br)c1F', 'CN=C/C(=N\\OC[C@@H](O)c1ccc(Br)cc1)C1CC1', 'Clc1ccc(Br)c([C@@H]([NH2+]Cc2cnc3cc[nH]c3c2)C2CC2)c1', 'O=C(CNCc1c(F)cccc1F)N1C[C@H](O)[C@H](O)C1', 'Clc1cc(Br)c(N/C=C/c2ccccc2)c2[nH]ccc12', 'O=C(/C=C/c1ccno1)N[C@@H](c1ccccc1)c1ccccn1', 'O=C(/C(c1cccs1)[C@@H]1COCCN1c1cccs1)NCc1ccncc1', 'O=C(c1cn[nH]n1)N(CC(F)(F)F)[C@@H]1CC[C@H]([NH+]2CCCC2)C1', 'Clc1cccc(CCNC(=O)N[C@@H]2CCC[C@H](C3CC3)C2)c1', 'O[C@@H]([C@@H](Cl)CO)C(=O)OCc1ccccn1', 'O=C(CC1([O-])CCC1)NCCc1nc2ccccc2o1', 'Clc1ccccc1-c1noc([C@@H]2CN(Cn3cccc3F)C2)n1', 'O=C(CN(CCC(N)=O)C1CCCCC1)Nc1ccccc1', 'Clc1ccccc1-c1cncc(CN2CCC[C@H]2c2ccccc2F)n1', 'NS(=O)(=O)/N=c1/c(=N/C(=O)Cc2ccccc2)c(C2CC2)nn1-c1ccccc1', 'Clc1cccc2c(N3CCC[NH+](CC4CC4)CC3)ncnc12', 'Clc1ccc(CO)n1CCSc1ccc(F)cc1Cl', 'OCS1(Cc2ccccc2)CC[NH2+]CC1', 'Clc1ccc(-c2csc3[nH]nc(-c4ccccc4)c23)cc1', 'O=C(CC1(O)CCOCC1)N1CCN(Cc2cccc(C(F)(F)F)c2)CC1', 'NS(=O)(=O)/N=C(/NCc1ccccc1)NC1CCN(c2ccccc2)CC1', 'O=C(/C(c1cccs1)[NH+]1CCCCC1)c1cn(Cc2ccccc2)nn1', 'N[C@@H](C[NH+](C)CC(=O)N1CCOCC1)c1ccc(F)cc1', 'Clc1nnn(-c2ncn([C@@H](C)CCN3CCOCC3)n2)c1', 'O=C(COCC1CCN(S(=O)(=O)c2ccccc2)CC1)N1N=C[C@@H](OC2CC2)c2ccccc21', 'Oc1ccc(/C=C(/CO)C[NH+](C)CCCS)cc1NC(=O)c1ccccc1', 'OCc1n[nH]cc1O[C@H](CN1CC[NH2+]CC1)[C@H](Cc1ccccc1)C(F)F', 'OC1(NC(=O)CN2CSc3ccccc32)CCCC1', 'CNS(=O)(=O)c1c(C)[nH]c(C(=O)Cc2ccccc2)c1C', 'Oc1ccc(/C=N/O[C@H]2CC(=O)N(c3ccccc3)C2)cc1Cl', 'CNC(=O)N(Cc1ccccc1OC)[C@@H](C)c1nccn1C', 'O=[N+]([O-])c1c([N+](=O)[O-])cnc2nc(C[S@@](=O)Cc3ccncc3)[nH]c12', 'Cc1noc(C)c1CCC(=O)N(C)Cc1ccccc1', 'Clc1ccsc1/N=N\\N1CCN(c2ccccc2F)CC1', 'Clc1ccccc1Nc1ccc2nnc(-c3c(F)cccc3Cl)n2c1', 'Oc1ccc(F)cc1-n1nc(C(C)(C)COC2CCCCC2)c2ccccc21', 'Clc1ccccc1C1(O[C@H]2CCS(=O)(=O)C2)CCCC1', 'Clc1ccc([C@@H](O)[C@@]2(CBr)CCOCC2)cn1', 'O=C(c1ccsc1COc1ccccc1)N1CCCc2ccccc21', 'Clc1ccccc1CNCCc1ccn(-c2ccccc2)n1', 'OCc1n[nH]cc1[C@H](CCO)c1ccc(Cl)cc1', 'O=C(Cc1coc2ncccc12)NCCn1nc(-c2ccccc2)ccc1=O', 'Oc1ccc([C@H](O)[C@@H](NC(=O)NC2CCCC2)c2cccs2)cc1', 'Clc1ccccc1OCc1ccn(Cc2ccccc2Cl)n1', 'O=[N+]([O-])c1c(-c2c[nH]nn2)cnnc1NCc1ccc(OC(F)(F)F)cc1Cl', 'Clc1cc(/C=C/N=C(\\c2ccccn2)c2ccccc2)cc(Cl)c1', 'Clc1ccc(C(C)(C)CN2CCNC[C@H]2c2ccccc2)cc1', 'Clc1cccc(F)c1COc1cccc(Cl)c1OCCC1CC1', 'Clc1nncn1CCN[C@H](c1ccccc1)c1ccccc1F', 'Clc1ccccc1OCC[NH2+]Cc1cn(-c2ccccc2)nn1', 'ClCc1ccn(C[C@@H]2CCCCN2S(=O)(=O)c2ccc(F)cc2)c1', 'Clc1ccccc1CN(c1ccccc1)S(=O)(=O)c1ccc(Cl)cc1', 'Clc1ccc(SCCCC(=O)N2CC[C@H](c3ccccc3)C2)[nH]1', 'O=[N+]([O-])c1c(F)cccc1NC1CC[NH+](Cc2ccc(Cl)cc2)CC1', 'Oc1ccc(Oc2cn(Cl)nc2-c2c[nH]c3ccccc23)cc1', 'O=C(c1ccco1)N1CCN(c2cc[nH+]cc2)CC1', 'C=C1c2ccccc2C(=O)O[C@H]1CC(=O)N1CCC2(C1)OCCO2', 'Clc1cc(Cl)c(Cn2c(=O)[nH]c3ccccc32)c(Br)c1Br', 'O=C(CCc1ccc[nH]1)Nc1c(F)cccc1F', 'C[C@@H](CNC[C@H]1CCCCC[NH2+]1)S(=O)(=O)c1nc(C2CCCCC2)no1', 'COc1nc(NC[C@H]2CCCO2)ccc1S(=O)(=O)NC1CCC(C)CC1', 'OC[C@@]1([NH3+])CC[C@@H](CNc2ccccc2Br)C1', 'C[C@@H](C[C@@H]1CCCC[NH2+]1)Nc1ccccc1Br', 'O=C(C1=c2ccccc2=[NH+][C@H]1N1CCOCC1)N1CCc2nccnc2C1', 'Clc1ccc(Br)cc1OCCOCc1ccccc1', 'O=[N+]([O-])c1ccncc1[C@@H]1CN(Cc2ccncc2)C1', 'ClCc1ccnnc1NC[C@H]1COc2ccccc2O1', 'COCc1c([O-])n2cc(-c3ccccc3)cc(-c3ccc(Cl)cc3)c2n1', 'O[C@@H]1[C@@H](C[NH+]2CCC[C@H](CSc3cc4ccccc4[nH]3)C2)CCCN1', 'Oc1ccccc1[C@H](Cl)c1ccc[nH]c1=O', 'O=[N+]([O-])c1cccc2[nH]ccc12', 'O=[N+]([O-])c1cccc2[nH]ccc12', 'O=[N+]([O-])c1cccc2[nH]ccc12', 'O=[N+]([O-])c1cccc2[nH]ccc12', 'O=C(c1c[nH]cn1)N(CCNc1ccc(OC(F)(F)F)cc1)C1CC1', 'O=C(Cc1nnc(-c2ccc(Br)cc2)o1)Nc1ccccc1F', 'Oc1ccccc1C1(O)CCCCC1', 'OC[C@@]1([NH3+])CCc2ccccc21', 'O=C(c1cnn(C2CCCC2)c1)N1CC[C@H](C(=O)[O-])c2ccccc21', 'Clc1cccc(F)c1[C@@H](CCn1cccn1)N[C@@H](c1ccccc1)c1ccccc1', 'CNS(=O)(=O)[C@@H](C)Cc1ccc([N+](=O)[O-])cc1', 'OC[C@@H](CO)N1CCC(c2ncc[nH]2)CC1', 'Nc1nnn([C@H](CCO)C2CC2)n1', 'Clc1nncn1/C(CO)C[C@H]1CCC[C@@H]1O', 'ClC[C@H](O)[C@@H](n1cncn1)C1CC1', 'COc1ncncc1C[NH+]1C[C@H](C(N)=O)[C@@H]2CCCC[C@@H]21', 'CN[C@H](NS(=O)(=O)c1ccc(C)c(F)c1)c1ccc(F)cc1', 'O=C(/N=C/CCc1ccccc1Br)Nc1ccc2c(c1)CCC2', 'O=C([C@H]1CO[C@H]1c1ccccc1)N1CCCN(c2nc3ccccn3[nH+]2)CC1', 'O=C([C@H](Sc1ccccc1)c1ccc(F)c(Cl)c1)NC1CCOCC1', 'O=C(Cc1cn(CC2CCC2)c(=O)[nH]1)Nc1cn[nH]c1', 'Nc1nnnn1CC[NH+]1CCN(Cc2ccccc2)CC1', 'O=C(CS(=O)(=O)CCC1CCCCC1)NNc1ccc2c(c1)COC2', 'Clc1ccccc1Oc1cc(NC(=S)Nc2ccccc2)ccc1Cl', 'ClC(=S)[C@@H](c1ccccc1)N1CCCc2ccccc21', 'Clc1ccc(-c2ccccc2)nc1SCC1CCCC1', 'O=[N+]([O-])c1ccn(CC[NH+]2CCC[C@@H]2c2ccc(O)cc2)c1', 'Oc1ccc(O[C@@H](c2ccco2)C(=O)NCc2ccccc2)cc1', 'ClCc1cn(C2CCN(c3ncccn3)CC2)cn1', 'ClCc1ccn(-c2ccc(C3CCCCC3)nn2)n1', 'Clc1ccccc1Cc1cn(C[C@@H]2CCCC[C@H]2[NH3+])nn1', 'C[C@@H](NC(=O)CCl)[C@H]1CCc2c([nH]c3ccccc21)c1ccsc13', 'O=C(/C=C1CC=C(c2ccc(F)c(F)c2)C1)[C@H]1COc2ccccc2C1', 'O=C(c1cn2ccsc2c1N)c1cc(Br)ccc1OCc1ccccc1', 'O=[N+]([O-])c1c(Br)scc1C(=O)NCCNC(=O)c1ccccc1F', 'O=[N+]([O-])c1ccc(S(=O)(=O)NCCCC(=O)N2CCCC[C@H]2Cn2cccn2)cc1', 'O=C(c1c[nH]c(-c2cccs2)n1)N(Cc1ccccc1)Cc1ccccn1', 'CNC(=O)COc1c(Cl)cccc1NC(=O)N1CCSC(C)C1', 'Clc1ccccc1O/N=C/C(=O)Nc1ccc2[nH+]ccc(C3CC[NH+]3C)c2c1', 'O=C(/C=C/O)C[C@H](c1ccccc1)NC(C)(C)C', 'Clc1cccc(S[C@@H]2C[NH2+]C[C@H]2CO)c1', 'Clc1cccc(C[NH2+][C@H]2CC[C@@H](S)C2)c1', 'NS(=O)(=O)N[C@@H]1CCN(S(=O)(=O)N(C)Cc2cccc(F)c2)C1', 'Clc1nnnn1/C(CCC(F)(F)F)N[C@H]1CC[C@@H](SCC(F)(F)F)C1', 'O=C(Cc1c[nH]nc1C(=O)NC[C@H]1CN2CCCC[C@@H]2CO1)Nc1nccs1', 'Oc1cccc(F)c1C[NH+]1CCC(OCc2ccccc2F)CC1', 'COc1ncnc(NCCc2ccccc2)c1Br', 'O=C(CSC(=S)[C@@H]1CCCC[NH+]1C1CC1)Nc1cncc(-c2ccccc2)n1', 'N#C[C@@]1(C2CC[NH+](CC(=O)N[C@H]3CCOc4ccc(F)cc43)c3[nH]cnc3C2)CC1', 'Oc1cccc(Br)c1C[NH2+][C@@H]1CCCc2ccccc21', 'O=[N+]([O-])c1cccc2c1/C(=N/CCCOc1ccccc1)CCCC2', 'Clc1ccc(CNC[C@H]2CCCN(C(=O)[C@@H]3CCCO3)C2)cc1', 'O=C(/C(CO)Cc1ccccc1OC(F)F)CCc1ccsc1', 'COc1nn[nH]c1C(=O)NC[C@@H](CCO)c1ccccc1', 'Clc1ccccc1NCC[NH2+]Cc1ccccc1', 'O=C(/N=C/c1ccccc1)c1cccc(C(=O)N2CCOc3ccccc3C2)c1', 'Clc1ccc(COCC[C@@H]2COc3ccccc32)cc1', 'Clc1ccc(Br)cc1SCCC(F)(F)F', 'O=C(/N=C\\c1occc1OCc1ccc(Cl)cc1)C1CCCC1', 'O[C@@H]1[C@@H](CNC2CCCCC2)CCCN(Cc2ccco2)C1', 'COc1nnn(CC2CCCC2)c1[O-]', 'ClC[C@H]([NH2+]Cc1ccccc1)[C@@H]1CCCC[C@@H]1O', 'Clc1cc(N2CCOCC2)ccc1OCc1cc[nH+]c2ccccc12', 'O=C(COCc1ccccc1)NC[C@H]1CCC[C@@H](O)C1', 'O=[N+]([O-])c1c([O-])c2ncn(CCCc3ccc(F)cc3)c2nc1N', 'O=C(c1cn[nH]c1-c1ccccc1)N1CCc2cc(S(=O)(=O)N3CCCC3)ccc21', 'CNS(=O)(=O)[C@@H]1CC[NH+](C2CCN(Cc3cccc4ccccc34)CC2)C1', 'Clc1ccccc1-c1cnnc(O[C@@H]2CCOC3(CCCC3)C2)n1', 'Clc1ccc(-c2nonc2)nc1O[C@@H]1CCOC2(CCCCC2)C1', 'Clc1cccc(F)c1NCc1ccc(F)cc1', 'Clc1ccccc1Oc1cc(F)c(Cn2ccc3[nH]ccc32)cc1', 'OC[C@@]1([C@H]([NH3+])Cc2ccccc2)CCOC1', 'Clc1ccccc1C[NH+](Cc1ccccc1)CC1OCCO1', 'Oc1ccccc1[C@H](N=C(CO)c1c[nH]c2ccccc12)c1cccs1', 'N=CC[NH+]1CCCC[C@@H]1c1ccc(S(=O)(=O)C(F)F)cc1', 'Clc1ccc(C[NH+](CCc2ccco2)[C@@H]2CCCc3cc(Cl)ccc32)cc1F', 'Clc1nnnc(O[C@@H]2CCCN(c3ccccc3)C2=O)c1Br', 'ClCc1ccn([C@H](OC(F)F)C2CCCCC2)c1', 'Clc1ccc(C[NH2+][C@H]2CCC[C@@H]2[C@@H]2CCCC2)cc1', 'Clc1ccc(C[NH2+]C[C@@H]2C[C@H]3CC[C@@H]2C3)cc1', 'CNc1[nH+]ccc(N)c1S(=O)(=O)NNc1cccc2ccccc12', 'Clc1ccccc1Nc1cnc(CCN2CCOCC2)[nH]1', 'ClC[C@H]([NH2+]Cc1ccccc1Br)c1ccsc1', 'Clc1cccc2cc(F)[nH]/c(=N/CCCNC(=O)c3ccc(OC(F)F)cc3)c12', 'OC[C@H](c1ccccc1)[NH+]1CCN(c2ccccc2)CC1', 'OCSc1ccccc1C(=O)NCc1ccccc1Oc1ccccc1', 'Clc1ccccc1C(CO)(Cl)[C@H](O)c1ccccc1', 'Clc1cccc2cc([C@H]3CCCCCN3C(=O)Cn3cncn3)ccc12', 'Clc1ccccc1O/N=C(\\[O-])c1c[nH]nc1-c1ccccc1', 'Clc1ccc(Cc2noc(CCC3CCCCC3)n2)o1', 'Clc1ccc(-c2nnnn2Cc2ccccc2)cc1', 'O=C([C@H](Nc1ccccc1)[NH+]1CCSCC1)c1ccccc1', 'O=C(Cc1cc(F)ccc1F)N1CCC[C@@H]1c1cccc(F)c1F', 'O=C(/C=C1/C[C@H](c2ccncc2)C(=O)Nc2c1CCCC2)N1CCOCC1', 'Clc1ccc(-c2nnnn2[C@@H](O)c2ccccc2Br)cc1', 'O=C([O-])[C@H](Cc1ccc(/N=C/c2cccc(OC(F)F)c2)o1)c1ccccc1', 'Oc1ccccc1/N=C/c1ccccc1COc1ccccc1Cl', 'Clc1cccc(Br)c1NCc1ccccc1Oc1ccccc1', 'O=C(/N=C/c1cc(-c2ccc(F)cc2)n(-c2ccccc2Cl)n1)NCC1(O)CCOCC1', 'Clc1nncc(N[NH+]2CCC[C@@H](C(=O)N3CCCCC3)C2)c1Cl', 'O=C(CCc1ccccc1)Nc1ccc(-n2ncn(C)c2=O)cc1', 'Oc1ccc(Oc2nc(COc3ccccc3)no2)cc1', 'Nc1cnnn1-c1ccccc1/C=C/C(=O)c1ccc(Br)cc1', 'Nc1nnnn1CC(=O)NCc1ccccc1OC(F)F', 'COc1nc(F)ccc1CC(=O)O[P@](=O)(=O)Cc1ccccc1', 'O=C(c1c[nH]nc1-c1ccc(F)cc1)c1cc(S(=O)(=O)N2CCCCC2)ccc1O', 'Nc1n[nH]nc1COC(=O)[C@@H](C)N1CCCCCC1=O', 'Clc1ccc([C@@H](Br)C[NH+]2CCC(C(N)=O)CC2)s1', 'O=C(CN(CCn1cccn1)c1ccccc1)NC[C@@H]1CCOC1', 'Clc1ccccc1-c1n[nH]c2cn[nH]c(=O)c12', 'Oc1ccc([N+](=O)Cc2nc(C3CC3)no2)c2ccccc12', 'O=C([O-])/C=C/c1ccccc1OCC1CC1', 'O[C@@H](Sc1ccccc1)C(=O)N1CCCCC1', 'O=C([O-])[C@@H]1CC(=O)N(CCSc2ccccc2)C1', 'Clc1ccc(CO/N=C/CCc2ccc3c(c2)OCO3)c(Br)c1', 'NS(=O)(=O)N(C)C1CCC(Oc2ccc(N(C)C)cc2)CC1', 'Clc1ccccc1C(C(F)(F)F)[C@@H]1CCN(c2ccccc2)C1=O', 'Clc1ccccc1NC(=O)[C@H](Cc1c(Cl)cccc1Cl)Nc1ccccc1O', 'Clc1cccc2cc(-n3ccnc3)n(-c3ccc(Cl)cc3)c12', 'Clc1cccc(/N=N/CCN2CCN(c3cccc(F)c3)CC2)c1', 'Oc1ccc(Oc2ccccc2C[NH2+]C2CC[NH2+]CC2)cc1', 'Clc1ccc(Br)c(SC2CCCC2)n1', 'Clc1cccc(CCn2ccc(C3CCC3)n2)c1', 'Clc1cccc(/C=C/CCn2cccc(C(F)(F)F)c2=O)c1', 'Clc1ccsc1[C@H](O)CNC(=O)N[C@H]1CCCc2ccccc21', 'O=C(c1cnc([C@H]2CCC[NH+](C/C=C\\c3cccs3)C2)o1)N1CCCC1', 'O=C(C[NH+]1CCC[C@@H]1c1ccccc1)Nc1nnc(C2CCCCC2)s1', 'Cc1c([C@@H](C)CCCCO)c(=O)n2c(-c3ccc(Br)cc3)n[nH]c2nn1C', 'Clc1ccccc1CNc1n[nH]c(-c2ccccc2)n1', 'Clc1cccc2ccc(SCCC[NH+]3CCC[C@@H](n4cccn4)C3)nc12', 'O=[N+]([O-])c1cc([C@H]2SCCCN2C(=O)COc2ccc(Cl)cc2)ccc1O', 'ClCc1noc(C[NH2+]Cc2cnn(Cc3ccccc3)n2)n1', 'COc1nn[nH]c1C(=O)C[C@H]1CCCN(c2ncccc2Br)C1', 'Clc1ccccc1Oc1cnc2n1C[C@@H](OCc1cccnc1)C2', 'Nc1c(/N=C/c2ccc3c(c2)OCO3)ncnc1N[C@H]1CCS(=O)(=O)C1', 'Oc1ccccc1[C@@H]([NH+]c1c(Br)cccc1Cl)[C@@H]1CCCO1', 'ClCc1ncoc1[C@@H]1CCC[NH+](Cc2ccc(F)cc2)C1', 'C[C@@H](NS(=O)(=O)c1ccc2ccccc2c1F)C(=O)NCCCc1ccccc1', 'ClC[C@H](OCCO)C1CCN(C(=O)c2ccco2)CC1', 'O=C(CSCC1(O)CCOCC1)c1cccc(F)c1', 'Cc1sc(NCCN(C)C)nc1S(=O)(=O)N1CCN(c2ccc(F)cc2)CC1', 'Clc1nnnc(Oc2c(Cl)cccc2OCC2CC2)c1Cl', 'Clc1nnnc(COc2ccccc2Cn2cccn2)n1', 'Clc1cc(CNC[C@@H](c2ccc3c(c2)OCO3)c2ccccc2)ncn1', 'O=C([O-])CS[C@H]1[NH+]=c2ccccc2=C1C1CC1', 'Clc1nncc(CN2CCC[C@@](O)(CCn3cnc4ccccc4c3=O)C2)c1', 'Clc1ccc(CN[C@@H]2CCC[NH2+][C@H]2c2ccc(F)cc2)cc1', 'Clc1ccc(CCC[NH2+][C@@H]2CCCc3cc(OCc4ccc(Cl)cc4)sc32)cc1', 'Oc1ccc(/C=N/O[C@@H](c2ccc(F)cc2)C2CC2)cc1', 'Clc1cccc([S@@]CC[C@@H]2CCC[NH2+]C2)c1', 'Clc1ccccc1[C@H](c1ccccc1)Sc1nnnn1-c1ccccc1', 'O=C([C@H](CC(F)(F)F)N1CCCS1(=O)[O-])N1CCc2ccccc21', 'C[C@@H](c1ccncc1)C(=O)N1CCCC(Cn2cc(Cl)cn2)C1', 'Nc1c(/C=N\\N(CCCC#Cc2ccccc2)c2ccco2)cc2ccccc2n1', 'Clc1ccccc1-c1noc(C(=O)N2CCC(Cc3ccco3)CC2)n1', 'Clc1ccsc1/CCNC(=O)NCCCN1CCCCC1=O', 'Clc1ccc(Br)cc1/N=C(\\CCn1cccn1)N1CC[NH+]2CCCC[C@@H]12', 'O=C(CN(C1CCCCC1)c1ccccc1)N(C1CCCCC1)S(=O)(=O)c1ccccc1', 'O=C(c1ccc[nH]c1=O)/C(=C/c1ccc(Cl)c([N+](=O)[O-])c1)OCC(F)(F)F', 'ClC(/N/N=C/c1ccccc1)C1CCOCC1', 'Clc1nncc(COc2ccccc2CN2CCOCC2)n1', 'COc1ncnc(N[C@@H]2CCO[C@H]2c2ccccc2F)c1[N+](=O)[O-]', 'ClCc1cnn([C@H]2CCCCC[C@@H]2C(=O)N2CCCC2)c1', 'N[C@@H]1[NH2+]CC(C)(C)[C@H]1C[C@@H](CO)c1ccccc1', 'Clc1ccc(CC[C@H]([NH3+])[C@H]2COc3ccccc3C2)o1', 'Clc1ccccc1OC1(F)CN(Cc2ccncc2)CC1', 'Oc1cccc2cc(/C=N/c3ncnc4[nH]ncc34)ccc12', 'Clc1ccc(Br)cc1S[C@H]1CCOCC1', 'O=C(Cc1nnnn1Cc1ccccc1)Nc1ccncc1', 'Oc1ccc(Oc2nc(N3C[C@H](C)O[C@H](Sc4ccc(F)cc4)C3)no2)cc1', 'Clc1ccccc1N1N(CCCc2nnc3cccsc23)CCC1', 'NS(=O)(=O)CC[C@@H]([NH2+]Cc1ccccc1)c1ccccn1', 'Nc1c(Cl)n2ncnc2n1CCS(=O)(=O)CC(=O)N1CCCC1', 'Oc1ccc(/C=C/C[NH2+]Cc2ccccc2O)cc1', 'ClCc1nonc1COC(c1ccccc1)c1ccccc1', 'Clc1ccccc1NCCn1nnc2ccccc21', 'Clc1cccc(CC[NH+]2CCCCCCC2)c1', 'Clc1cccc(CCC[NH+]2CCCCCC2)c1', 'Clc1ccc([C@@H](CC(=O)N2CCOc3ccc(Cl)cc32)N2CCOCC2)cc1', 'Clc1cccc([C@@H](N[C@H]2CC[NH+](Cc3ccco3)c3ccccc3)C2)c1', 'COc1nnnc(NC(=O)Cc2ccc(I)cc2)n1', 'NS(=O)(=O)NC[C@@H](c1ccco1)N1CCN(C(=O)[C@@H]2CCc3ccccc3C2)CC1', 'Clc1ccccc1N1N(CCOc2ccc(Br)cc2Cl)CCCC1', 'Clc1ccccc1OC1(NC(=O)C(=O)c2ccccc2)CC1', 'CC[S@](=O)[C@H]1CCCC[C@H]1NC(=O)C(=O)Nc1ccc2[nH]cnc2c1', 'Clc1cccc2c(N[C@@H]([C@@H]3CCCO3)[C@H]3CCS(=O)(=O)C3)cccc12', 'O=C(Cc1n[nH]n1C1CC1)Nc1ccc2oc(-c3cccs3)nc2c1', 'Clc1cc(Cl)nnc1N[C@H]1C[C@H]2CC[C@@H](C1)[NH2+]2', 'O=C([C@H](Cl)CCSc1ccccc1)NC[C@@H]1CCCO1', 'Nc1nonc1C[NH2+]C1CCN(CC(=O)c2ccccc2)CC1', 'Nc1[nH]ncc1CN(Cc1ccccc1[N+](=O)[O-])C[C@H]1CCCO1', 'Clc1ccc(-c2cc[nH]n2)nc1C[NH2+]CC[C@H]1OCCO1', 'O=[N+]([O-])c1c(Cl)nc(N2CCCC2)nc1Oc1ccc(Cl)cc1', 'Clc1cc(CN(c2c(Cl)cccc2Cl)SC2CCCC2)c2ncccn12', 'Clc1ccccc1C1(Oc2ccccc2)CC1', 'Oc1cc(O[C@@H](CO)c2ccccc2)ccc1-c1ccc(Cl)cc1', 'Clc1ccc(C[NH+](CC[C@H]2CCCCO2)c2ncccc2C(=O)[O-])cc1', 'Clc1ccccc1CO[C@H](CO)c1ccc(Cl)cc1', 'O=C(c1ccc(Br)cc1)N1CCC(OC(F)F)CC1', 'ClC[C@H](Br)Cc1ccccc1', 'OC[C@H](C(=O)N1CCN(c2nc(-c3ccccc3)cs2)CC1)c1cccs1', 'Clc1ccccc1-c1csc(C[C@@H]2C[C@@H]2c2ccccc2)n1', 'O=C(Cc1c(O)cccc1O)N[C@@H]1CCCC[C@H]1c1ccccc1', 'Clc1ccc(C[NH2+][C@H]2CCC[C@H](n3ccnn3)C2)cc1', 'ClCc1cn(C[NH+]2CCCC[C@H]2c2ccccc2)nn1', 'Clc1cc(Cl)nnc1NCc1cnn([C@@H]2CCC[NH2+]C2)c1C1CC1', 'Clc1nncc(C2CC2)n1', 'NS(=O)(=O)c1ccc(NC(=O)N2CCC(CS(=O)(=O)c3ccc(F)cc3)C(=O)C2)cc1', 'CNS(=O)(=O)c1ccc(F)cc1N[C@H]1CCC[C@@H](S(=O)(=O)N(C)C)C1', 'Clc1ccsc1/C=C/CC[NH2+]C(CC(=O)[O-])c1ccccc1', 'O=C(COc1ccccc1F)NCC[NH+]1CCc2ccccc21', 'O=[N+]([O-])c1ccn([C@H](CO)c2ccc(Cl)cc2)c1', 'Clc1ccc(Br)cc1NC[C@H](c1ccccc1)[NH+]1CCCC1', 'ClCc1nc(N2CCCCC2)c2nc(N3CCCCC3)nnc2n1', 'Clc1ccc(C[NH2+]C2CCSCC2)cc1', 'Clc1cccc(NC2[C@@H](N3CCC(c4ccccc4)CC3)C2)c1', 'Clc1ccccc1OCCCCn1cc(Br)cn1', 'Clc1ccccc1Oc1cc(Cl)ccc1C(=O)NC[C@@H]1CCOC1', 'Oc1ccc(F)c(F)c1SCCN1CC[NH+](Cc2ccccc2F)CC1', 'Clc1ccc(-c2nonc2C[NH+]2CCCC2)cc1', 'Clc1ccc(-c2nonc2C[C@@H]2CCCC[NH2+]2)cc1', 'Clc1ccccc1Oc1cc(O[C@@H]2CCOC3(CCOCC3)C2)nc2ccccc12', 'O=[N+]([O-])c1ccc[nH+]c1SCCF', 'Clc1ccccc1CNC[C@H](O)c1ccccc1', 'Clc1cccc(F)c1CNCc1ccccc1F', 'Clc1ccc([C@@H]([C@@H]2CCOC3(CCCCC3)C2)OC[C@H](O)CO)cc1', 'Clc1ccccc1NC(=O)[C@@H]1CC(c2ccccc2)N=[N+]1c1ccccc1', 'O=[N+]([O-])c1cccc(CNC2CCSCC2)c1', 'Clc1ccc(Br)cc1-n1cccc1C(=O)N1CCCC[C@H]1CO', 'CNS(=O)(=O)c1csc(C(=O)N[C@@H](C)C(=O)N2CCCCCC2)c1', 'Clc1ccccc1C(=O)OC1(c2ccc(Cl)cc2)CC1', 'Clc1cccc([C@@H](Sc2ccccc2)[C@H]2CCCO2)c1', 'Clc1ccc(Br)cc1-n1nnc2sc(-c3ccncc3)nc21', 'O=C(CN(c1cnc(C(=O)NCc2ccccc2)c(Cl)c1)S(=O)(=O)c1ccoc1)c1ccccc1', 'CNc1n[nH]c(=S)n1CC(=O)N1CCO[C@@H](c2ccccc2)C1', 'Clc1ccccc1OCc1ccc(N2CCOCC2)cc1', 'N=C1N(C[C@H]([NH2+]C)c2ccccc2F)C(=O)NC([S-])=NC1', 'O=[N+]([O-])c1c([O-])[nH]c(=O)n1CC(=O)Nc1ccc(Oc2ncccn2)cc1', 'Oc1cc(-n2cc(C[NH+]3CCN(c4ccccc4)C3=O)CC2)ccc1Br', 'Clc1nncc(S/C(=C/C(=O)[O-])COC(F)(F)F)n1', 'CCc1ccc2nc(N(Cc3ccco3)C(=O)c3ccc(O)c(F)c3)[nH]c2c1', 'Oc1cc(O)c(Oc2ccc(N3CCCC3=O)cc2)nc1NC[C@@H]1CCCO1', 'COc1ncc2n1CCC[C@@H]2[NH+]1C[C@@H]2CCCC[C@@H]21', 'CN[C@H](CSC)c1ccncc1', 'Clc1ccccc1NCc1ccc([S@](=O)C2CCC2)cc1', 'ClC(F)(CBr)c1c[nH]cn1', 'OC[C@H]1CN(Cc2ccccc2CO)C[C@@H]1c1ccco1', 'Clc1ccc(Cc2nnc(NC[C@H]3CCCO3)o2)cc1', 'O=[N+]([O-])c1c(Br)cc(Cl)cc1C(=O)N1CCO[C@@H](c2ccccc2)C1', 'O=C(c1ccsc1Cn1cnnn1)NC(c1ccccc1)c1ccccc1', 'Clc1ccccc1NS[C@H](c1ccccc1)N1CCOCC1', 'Clc1ccccc1NS[C@H](c1ccccc1)N1CCOCC1', 'Oc1cccc(Br)c1C[C@H]([NH2+]Cc1ccc(Cl)cc1)c1cccc(F)c1', 'COc1nnnn1CCNC(=O)c1ccc(Cl)c([N+](=O)[O-])c1C', 'Clc1nncc(-c2cn3c(cnn3-c3cccc(F)c3F)c2)c1Cl', 'Oc1cccc2cc([C@@H](CNS(=O)(=O)c3ccc4c(c3)OCO4)C(=O)[O-])c(=O)n(Cc3ccccc3)c12', 'C[C@@H](O)c1cccc(CS[C@@H]2CCCOC2)c1', 'CN[C@H](N[C@@H]1CCOC1(C)C)c1nnc2n1CCCCC2', 'Clc1ccc(CO)cc1SCC[C@H]1CCC[NH+](C(=O)Cc2ccccc2)C1', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(Br)cc1N1CCN(c2ncccn2)CC1', 'Oc1cc(O[C@@H](CN2CCOCC2)[C@@H]2CCC[NH+]2Cc2ccccc2)ccc1O', 'Oc1ccc(O[C@@H]2C[C@@H]2C(=O)O[C@H]2CCCc3ccccc32)cc1[N+](=O)[O-]', 'Nc1nccnc1SCC(=O)N[C@@H]1CCS(=O)(=O)C1', 'Oc1cccc2cc(C(=O)N3CCCCC3)c3nc(-c4ccccc4)c(=O)[nH]c3c12', 'O=C(COc1ccccn1)N[C@H](Cc1ccccc1)n1cncn1', 'O=C([C@H]1CC[NH+](CCC2CC2)C1)N1CCN(c2ccc(Cl)cc2)CC1', 'O=C(CCS(=O)(=O)Cc1ccc(Cl)cc1)NC1CCCC1', 'Nc1c(Oc2ccc(C(N)=O)nc2CO)nc2ccccc2c1Cl', 'CNS(=O)(=O)[C@@H]1CCOCCN1C(=O)/C=C/c1ccc(Br)cc1', 'O=C(c1cnn(C[C@H]2CCCO2)c1)N1CCCCC1', 'Clc1nncc(F)c1NC[C@H]1CCCO1', 'Clc1ccccc1Oc1cc(NC(=O)CC2=CCCCC2)ccc1F', 'Clc1ccc(Br)cc1OCC[C@@H]1CNC(=O)N1Cc1ccccc1', 'ClC[C@@H]1O[C@H]2OCCN2[C@@H](c2cccnc2)O1', 'Clc1nncn1[C@@H]1CCOC2(CCCCC2)C1', 'Clc1nnc(N[C@H]2CCOC3(CCOCC3)O2)cn1', 'Oc1cc(-n2nncc2CN2CCN(Cc3ccccc3)CC2)ccc1[N+](=O)[O-]', 'Oc1ccc([N+](=O)[O-])c(C[NH+]2CCC(Oc3ccccc3)CC2)c1', 'Clc1cccc([S@@]Cc2cccc3ccccc23)c1', 'COc1nn[nH]c1/C=C/C(=O)N[C@@H](CO)c1ccccc1', 'Clc1ccccc1NC1CC[NH+](C2CCCC2)CC1', 'Clc1ccccc1CO[C@H]1CCCOC1', 'O=C(c1ccccc1)c1ccc(Br)cc1OCC[NH+]1CCCC1', 'Oc1cc(O[C@@H]2Oc3ccccc3OC2)ccc1NCc1ccc(Cl)c(Cl)c1', 'O=C(/C=C\\Nc1nccs1)NCCc1ccccc1', 'Oc1ccc(O[C@@H](c2ccco2)[C@H](CO)C2CC2)cn1', 'CC[C@@H](OC(=O)CN1C(=O)/C(=C/c2ccc(Oc3ccccc3)cc2C)SC1=S)C(=O)[O-]', 'Clc1cc(Br)cn1N[C@@H]1C[C@H](O)CN1C(=O)Nc1cccc2ccccc12', 'O[C@@H](S[C@@H]1CCOC1)c1ccccc1', 'O=C(/C=C/c1c(Cl)ccc2ccccc12)N1CCCC[C@@H]1C[C@@H](O)CO', 'Clc1ccc(Br)cc1OCC[C@H]1COc2ccccc2O1', 'O[C@@H]([C@@H]1Cc2ccccc2O1)C1(c2ccccc2)CCOCC1', 'Clc1nnn(-c2ncc3nnn(-c4ccccc4)c3c2)c1N', 'ClC(/C=N/O)c1c[nH+]ccc1N1CCOCC1', 'Clc1ccc([C@@H](Cl)C[C@@H]2COc3ccccc3C2)cc1', 'Clc1ccc(-c2nn(-c3ccc(Br)cc3)c(=O)n2Cc2cccc(F)c2)cc1', 'O=C(CNC(=O)C(F)(F)F)N[C@@H](C(=O)c1ccc(Cl)cc1)C1CCCC1', 'OCS[C@@H]([NH2+]C[C@@H]1CCCO1)c1nccn1C1CCCC1', 'O=C(/C=C1\\C(=O)Nc2ccccc21)c1ccccc1Cn1cccn1', 'OC[C@H]1[C@H](Cc2ccc(O)cc2)NC(=O)N1Cc1ccc(F)cc1', 'O=C([O-])/C=C(\\Nc1ccc(Br)cc1)N[C@@H]1CCOC1', 'Cc1ccc(S(=O)(=O)N2CCCC[C@@H]2C(=O)NOc2cncc(C(F)(F)F)n2)cc1', 'Nc1[nH]nc(SCCOc2ccc(F)cc2)c1[N+](=O)[O-]', 'CNC(=O)C[NH+]1CCN(c2ccccc2Br)CC1', 'O=C(CSCCO)N1CCC[C@H]1c1cccc(Cl)c1', 'Clc1cccc(F)c1CCNC[C@]1(O)CCSC1', 'O=[N+]([O-])c1c(-c2nsc(-c3ccsc3)n2)nc2ccccc2c1[O-]', 'OCS/C=C/Nc1cc([N+](=O)[O-])ccc1N1CCCCC1', 'Clc1ccc(-c2ccccc2CCO)cn1', 'Clc1cc(/C=Cc2[nH+]cn(-c3ccccc3)n2)ccc1NN1CCCC1', 'Clc1cc(CNC[C@H](c2ccccc2)c2ccccc2)c(F)cn1', 'O=C(c1ccc[nH]c1=O)N1CCO[C@@]2(CCS(=O)=C2CO)N1', 'O=C(Cc1ccco1)NC[C@H](CC(F)(F)F)c1ccccc1', 'Clc1cc(/N=C/c2sc3ccccc3c2C[NH+]2CCCC2)cc(O)c1O', 'Clc1cccc(F)c1CSc1cccc(Cl)c1', 'Clc1ccc(Br)cc1SCc1ccc(F)cc1', 'OC[C@H]1ON=C2CCCN2/C1=C/c1ccnc([N+](=O)[O-])c1', 'Clc1cc(CNCC[NH+]2CCC[C@H]2C[NH+]2CCCC2)cnc1C1CC1', 'Clc1nnnc(N2CCOCC2)c1NCC1OCOC1', 'O=C(c1cc([N+](=O)[O-])ccc1N1CCCCC1)N1CCC[C@H](c2nccs2)C1', 'Clc1cc(Br)c(Br)cc1NC[C@@H]1COc2ccccc2O1', 'O=C(CN(c1ccccc1)S(=O)(=O)/C=C/c1ccccc1)NCc1ccccc1', 'OC[C@H]1CCCC[NH+]1Cc1cccc(OCc2ccccc2)c1', 'Clc1ccc([C@@H]([C@@H](O)[C@H]2CCCO2)Sc2ccccc2)nc1', 'Oc1ccccc1[C@H]([C@@H](O)c1ccncc1)NC1CC1', 'O=C([C@H](c1ccc[nH]1)[NH+]1CCCCC1)N1CCN(c2nc3ccccc3nc2S(=O)(=O)C2CCCC2)CC1', 'ClCc1ccc[nH]c1=O', 'ClCc1ccc[nH]c1=O', 'Clc1ccccc1OC1C=C[C@H](OC2CCCCC2)[NH+]1', 'Oc1ccccc1OSC[C@H]([NH3+])CO', 'Oc1ccc(O[C@H](CSc2ccccc2)C(=O)NC2CC2)cc1', 'Clc1ccc(CN[C@@H](c2nc3ccccc3s2)c2ccccc2)cc1', 'ClC[C@H](CS(=O)(=O)c1cc(F)ccc1F)Cn1cccn1', 'Clc1ccccc1OCCNc1ncccn1', 'O=C(c1cccnc1N1CC[NH+](Cc2ccc(Cl)cc2)CC1)N1CCCCC1', 'Clc1ccccc1-c1cscc1-c1c[nH]c2ccccc12', 'Oc1ccccc1O[C@@H]([C@@H]1CCCN(c2ncccn2)C(F)(F)F)C1', 'Clc1nncnc1N[C@@H](c1ccccc1)c1ccccn1', 'Clc1ccccc1Oc1cncc(Oc2ccc(Cl)cc2)n1', 'Clc1ccccc1Oc1cnc(Nc2ccccc2F)cn1', 'Clc1ccccc1Nc1cnc(Oc2ccc(Cl)cc2)nc1', 'Clc1ccccc1Nc1ccnc(Oc2ccccc2)n1', 'Clc1ccc(-c2nonc2NC2=N[C@H]3C4CCCCC43)[C@H]2C1', 'O=C(c1c[nH]nc1CNCc1ccc(Br)cc1)N1CCN(c2cccc(Cl)c2)CC1', 'O=C(c1ccc(C(F)(F)F)cc1)N1CCC(Oc2ccc(-c3ccccc3)s2)CC1', 'Oc1ccc(O[C@@H](Cc2nc3cc(Cl)ccn3c2C(F)(F)F)C2CCCC2)cc1', 'ClC[C@H](C(=O)NCc1ccc([N+](=O)[O-])cc1)N1CCCSCCC1', 'Clc1ccccc1OCCCOc1ccc(Cl)cc1', 'COc1nn(CC[NH+]2CCC[C@H](C(=O)NC)C2)c(C)c1Br', 'Clc1ccccc1C[NH+]1CCC(CNc2ccc(Cl)cc2[N+](=O)[O-])CC1', 'O=C(/C=C\\CN1CCSC1=O)c1ccc(Cl)c(Cl)c1', 'CN(c1ccn(-c2ccccc2Cl)n1)C1CCCC1', 'N=CC1CCN([C@H](C(=O)N2CCCC2)c2ccccc2)CC1', 'ClCc1ccn([C@H]2CCC[NH2+]C2)c1', 'Oc1ccccc1/C(Cl)Cc1ncccc1OC1CCCCC1', 'N[C@@H](NC(=O)c1ccccc1OCc1ccco1)C1CCC1', 'Clc1ccc(-c2ccccc2)cc1C[NH2+]C[C@H]1CCS(=O)(=O)C1', 'Clc1ccccc1OCc1ccn(CCCOC(F)F)n1', 'O=C(CNCc1c(-c2ccccc2)noc1CO)NCc1ncoc1C1CC1', 'O=C(CN(CC1CC1)C1CC1)Nc1cccc2cccnc12', 'Nc1c(C(=O)N[C@H](CO)c2ccc(Cl)cc2)cnn1-c1ccccc1', 'O=C(c1cn[nH]n1)N1CCC[C@H](c2[nH+]ccn2Cc2cccc(F)c2F)C1', 'Clc1nnnc(N2CCC[C@@H](Cc3ccccc3)C2)n1', 'Clc1ccccc1NCc1nnc2n1CCCCC2', 'Clc1ccccc1-c1noc(CN2CC[NH2+]CC2)n1', 'Clc1cccc2nc(NC[C@@H]3CCCCO3)ncc1-2', 'Clc1nnnc(S[C@H]2CCO[C@@]3(CCC[C@@H]3c3ccccc3)C2)n1', 'ClCc1n[nH]c(S[C@@H]2CCCC[C@H]2Cn2ncn(CC3CC3)c2=O)n1', 'O=C([C@H](Sc1ccccc1F)N1CCC(c2ccccc2)(C(=O)[O-])CC1)n1cccc1', 'Oc1ccc([C@H](c2cc3ccccc3[nH]c2=O)c2cc[nH]n2)cc1', 'Clc1ccc(COCOc2ccccc2F)cc1', 'O=C(CCS)c1ccccc1', 'C[C@@H](OCC[NH3+])[C@H]1C[NH2+]CC[C@@H]1c1c(F)cccc1F', 'Clc1ccc(Br)cc1N(C1CCOCC1)C1CCCCC1', 'CC(C)[C@H](C(=O)Nc1ccccc1)N(C)C(=O)c1ccc2c(c1)OCCO2', 'Oc1ccc(/C=N/Nc2ncnc3ccccc23)cc1', 'Oc1cc(OCCn2cnc(-c3ccccc3)n2)cc(Cl)c1OCc1cccs1', 'Cc1cccc2c1[C@@H](Cc1cccnc1)C2CC[NH3+]', 'Clc1ccccc1OCCOc1ncccc1F', 'O=[N+]([O-])c1c(F)ccc(F)c1C(=O)NCCC[NH+]1CCCCC1', 'O=C([O-])[C@H]([C@H](O)Cc1ccc(CC(F)F)cc1)N1CCOCC1', 'Clc1ccc(-c2cccs2)o1', 'Clc1ccccc1CN(C1CC1)S(=O)(=O)c1ccccc1Cl', 'N=Cc1ncoc1-c1ccccc1OC1CCN(CC(F)(F)F)CC1', 'Oc1ccc(/C=N\\Nc2nc(Cl)ccn2)cc1', 'Clc1ccccc1OCc1csc2ccccc12', 'O=C(C1=N[C@@H]2[C@H](c3ccccc3)C[C@@](C)(C)[C@H]12)c1ccccc1', 'Clc1ccccc1N1NCC2(CCCCC2)O/C1=C/c1ccc(F)cc1', 'O=C(CNS(=O)(=O)C1CCCC1)Nc1ccc(F)c(F)c1', 'CNS(=O)(=O)CC(=O)N[C@H](C)c1ccc(OC)c(F)c1', 'CCC[NH2+]Cc1cc(S(C)(=O)=O)ccc1[N+](=O)[O-]', 'O=C(/N=c1\\onc2n1CCC2)N1CCN(c2ccccc2)CC1', 'Clc1ccc(C[NH2+]c2ccnc(-c3ccco3)n2)cc1', 'O=C(Cc1noc2nc(-c3ccccc3)oc12)Nc1ccc(F)cc1', 'Oc1ccccc1O[C@@H](c1ccccc1)c1c(N)cnn1C', 'Clc1cccc(F)c1-n1cc(Cl)ccc1=O', 'Oc1ccc(/C=N/OC[NH+]2CCC[C@@H]2c2nc3ccccc3s2)cc1', 'Clc1nnc(/N=C/c2ccc3c(c2)OCCCO3)c(-c2ccccc2)n1', 'ClC[C@H](Br)c1n[nH]cc1C(F)(F)F', 'Clc1ccc(Br)cc1S(=O)(c1ccc(F)cc1)C(=O)Nc1ccc(Br)cc1', 'Oc1ccc(Oc2cc3occc3nc2C(=O)N[C@H]2CC=CCCC2)cc1', 'O=C(COCCC[NH3+])Nc1ccccc1F', 'O=C([C@H]1COc2cccc3cccc(c23)CN1C(=O)n1ncc(Br)c1Cl)N1CCCC1', 'Oc1cccc(S[C@@H](c2nccn2-c2ccccc2)c2ccccc2)c1', 'Clc1ccc([C@@H](Br)[C@@](O)(c2ccccc2)c2ccccc2)cc1', 'Clc1ccc(Br)cc1O[C@H]1CCCN(C(c2ccccc2)C(F)(F)F)C1', 'Oc1cccc(Br)c1OCCCc1c(F)cccc1Cl', 'Clc1nncn1c1cc(Cl)ccc1N1CC[C@H](CN2CCOCC2)C1', 'Oc1ccc(O[C@H]2C=[NH+]N=C3CCC[C@H]32)cc1C[NH2+][C@H]1CCS(=O)(=O)C1', 'O=C(Cc1c(F)cccc1Cl)NN[C@H]1CCS(=O)(=O)C1', 'Clc1ccc(Br)cc1O[C@H](c1ccccc1)[C@@H]1CCS(=O)(=O)C1', 'O=C([O-])c1[nH]ncc1C(=O)N1CCN(c2ccc(C(=O)[O-])cc2)CC1', 'O=C(C1=c2[nH]ccc2=[NH+]C1)N1CC[C@@H](Nc2cccc(Cl)c2)C1', 'Clc1ccc(C[NH+]2CCC[C@]2(C(=O)[O-])[C@@H]2CCOC3(CCOCC3)C2)cc1', 'Clc1nnc(N[C@@H]2C[C@H]2[N+]2=NCCS2)nc1NCc1ccccc1', 'O=[N+]([O-])c1c(-c2cc(Cl)ccc2Cl)[nH]c2c1C[NH+](C[C@@H]1CCCCO1)CC2', 'NS(=O)(=O)Nc1cc([C@@H]2CCC[NH+]2CC2CC2)ccc1Cl', 'NS(=O)(=O)Nc1cc(Cl)ccc1C(=O)NC[C@@H]1CCCO1', 'ClCc1cnn([C@@H]2CCCC[NH+]2Cc2ccccn2)n1', 'OC[C@@H](N[C@H]1CCSc2ccccc21)[C@@H]1CCC(=O)N1', 'CN=C(c1ccccc1)c1ncn(-c2ccccc2)c1[N+](=O)[O-]', 'O[C@@H](c1ccc(Cl)cc1Cl)c1ccccc1F', 'Clc1ccccc1-c1n[nH]cc1CN1C[C@@H]2CC[C@@H](C1)C2=O', 'Clc1cc(F)c(N2CCC2)cc1NCC(=O)N1CCOCC1', 'CC(C)N(Cc1[nH+]ccn1C)S(=O)(=O)c1ccc2c(c1)CCN2C(=O)OC(C)C', 'O=C(c1ccsc1)N(C[C@H]1COc2ccccc21)c1ccc(C(F)(F)F)cc1', 'OC[C@@H](CNc1ccnc(Cl)c1)N[C@@H]1CC(CO)(c2ccccc2)n2ccccc2=N1', 'Clc1cccc(CC(=O)N[C@H]2CCCC[C@@H]2CO)c1F', 'Clc1cccc2cc(CNC[C@@H]3CCc4ccccc4O3)c(F)cc12', 'Oc1ccccc1OSC[C@@H](O)c1ccccc1Cl', 'Oc1ccc(/C=C(\\c2ccccc2)C(=O)Nc2cc(C(F)(F)F)c(Cl)c(Cl)c2)cc1', 'O=[N+]([O-])c1c(Cl)nsc1N[C@H]1C[C@@H]2C(=O)CCN(Cc3ccccc3)C[C@H]21', 'O=[N+]([O-])c1c(Cl)nnc2nc(NCc3ccccc3)[nH]c(=O)c12', 'O=C(c1cncc(Cn2cncn2)n1)Nc1ccccc1Cl', 'Nc1[nH]ncc1Nc1ncnc(Nc2ccc(Cl)cc2)n1', 'Clc1ccc(-c2n[nH]c(NNc3ccccc3Cl)n2)cc1', 'Clc1ccccc1Oc1ccccc1F', 'Clc1ccc(-c2n[nH]cc2CNc2ccc(Cl)c(F)c2F)cc1', 'CCS(=[NH2+])=c1/c(=N/C(=O)COC)sc2c(C)cc(C)cc12', 'Oc1ccc(F)cc1[C@H]1Oc2ccccc2C(=O)N1C[C@@H](O)C1CC1', 'Oc1ccc(F)cc1O[C@H](C(=O)NC[C@H]1CCCO1)c1ccccc1', 'CCNc1ccc(C(=O)N[C@@H](C)c2ccccc2Br)cc1OC', 'Clc1nncnc1[NH2+]C[C@@H]1C=C[C@H](C(=O)[O-])C1', 'ClCc1cn[nH]c1N1CCC[C@H]1c1ccccn1', 'O=C(/C(CN1CCN(c2ccccc2)CC1)c1ccccc1)S[C@H]1CCS(=O)(=O)C1', 'Clc1ccccc1-c1cscc1C[NH2+]Cc1ccccc1', 'Clc1ccc(C2C[C@](NC3CC3)/C(=C/Cl)C2)cc1', 'O=C(/C(c1cccs1)=N/NC(=O)[C@H]1C[C@@H]1c1cccc([N+](=O)[O-])c1)c1cccc(Cl)c1Cl', 'NS(=O)(=O)CCCNC(=O)Nn1cccc1', 'Clc1ccc(C(N)=N/S(=O)(=O)CCCC(=O)OC(F)(F)F)cc1', 'N=Cc1c(NC(=O)[C@]2(c3ccc(F)cc3F)NC(=O)C(=O)N2C)cccc1OC(F)F', 'CCN(C[C@@H]1CCOC1)C(=O)NCc1cccc(C[NH+]2CCCC2)n1', 'Clc1ccccc1-c1cscc1C[NH2+]Cc1nnc2n1CCC2', 'Oc1cccc(O[C@@H]2CCO[C@]2(Cc2ccccc2Br)C2CC2)c1', 'C[C@@H](N[C@@H](c1ccc(F)cc1)c1n[nH]c(N)n1)c1ccncc1', 'O=[N+]([O-])c1cccc2c1CCN(C[C@H](O)CN1CCc3cc(F)cc(F)c3C1)C2', 'Oc1ccccc1CC[NH+]1CCN(Cc2ccccc2)CC1', 'O=C(CNc1cnc2ccccc2n1)NCCn1cc[nH+]c1', 'Clc1ccc(-c2csc3[nH]c(=O)c(-n4cccn4)cc23)cc1', 'COc1nnnn1-c1cncc(NCc2ccc(C(F)(F)F)cc2Cl)c1', 'O=C(c1c[nH]cn1)N[C@H](CO)c1ccccc1F', 'Oc1cccc(Br)c1OCC1(O)CCCCC1', 'O=C([C@H]1[C@H]2C(=O)N(CCOCC(F)(F)F)C(=O)[C@H]12)c1cccs1', 'O=C(CCc1ccc([N+](=O)[O-])cc1)N1CCN(C(=O)c2sc3ccc(F)cc3c2C)CC1', 'Clc1ccc(-c2cc(-c3csc4ccccc34)[nH]n2)cc1', 'COc1ncncc1C(=O)Nc1ccnn1C1CCCCC1', 'CNC(=O)CNC(=O)C1CCN(c2ncnc3[nH]cnc23)CC1', 'O=[N+]([O-])c1ccn(CCNC(=O)Nc2ncn[nH]2)c1', 'Clc1cc(CNc2ccc(N3CCOCC3)cc2)nc(-c2ccncc2)n1', 'Clc1ccc(-c2cccnc2Cl)cc1', 'Clc1ccc(-c2cccnc2Cl)cc1', 'Clc1ccc(-c2cccnc2Cl)cc1', 'C[C@@H](Oc1ccccc1[N+](=O)[O-])c1nc(C(F)(F)F)ccc1Cl', 'Oc1cc(F)c(-c2ccco2)cc1N1CCC[C@@H]1C[NH+]1CCCC1', 'O=C([C@H](N[C@@H]1CCS(=O)(=O)C1)N1CCCC1)Nc1nccs1', 'O=C([C@H](CSc1ccccc1)c1ccccc1)N1CCCC1', 'Clc1ccc(-c2cccs2)cc1', 'Oc1cccc(Br)c1Cn1cccc(Br)c1=O', 'Clc1ccc(-c2nnnn2/N=C\\c2ccccc2Cl)cc1', 'Clc1ccc(-c2nnnn2/N=C/c2ccccc2Cl)cc1', 'Clc1ccsc1/C=N/N=C1\\S[C@H]2CS(=O)(=O)C[C@H]2N1CC1CC1', 'O=[N+]([O-])c1c([O-])c2[nH]cnc2n1C[C@@H]1C[C@@H]2CC[C@@H]1C2', 'C[C@@H]([C@@H](C)O)[NH+]1CCN(C(=O)Cc2csc(C(C)C)n2)CC1', 'Clc1ccc(-c2nonc2COC2CC2)cc1', 'Clc1ccsc1[C@@H]1CSCCS1', 'O=C(c1c[nH]cn1)[C@H]1CC(=O)N(Cc2ccc(Cl)cc2)C1', 'NS(=O)(=O)Cc1cc(NCC[NH+]2CCCCC2)ccn1', 'Clc1nncnc1N[C@H](CO)c1ccccc1F', 'Clc1ccccc1CNc1n[nH]c(Cc2ccc(Br)cc2)n1', 'CNS(=O)(=O)c1ccc(C(=O)[C@@H](C#N)C(=O)CC(C)(C)CC)cc1', 'CCOc1ccc(N2C[C@H](C(=O)NC[C@@H](Br)CCO)CC2)cc1', 'Clc1ccc(Br)cc1OCCN1CCN(c2ccccc2)CC1', 'O=C(c1c[nH]nc1C1CC1)c1cc(-c2ccc(Br)cc2)nc2ncnn12', 'ClC[C@@H]1Oc2ccccc2[C@@H]1Cc1c(F)cccc1Cl', 'O=[N+]([O-])c1cc(O)cc(C[NH+]2CC[C@H]3SCCC[C@@H]3N2)c1', 'CN=C/S(=O)(=O)Cc1ccc(OC(C)C)cc1', 'N#C[C@@]1([NH2+]C2CC2)CCC[C@@H]1CCSc1ccccc1', 'OCc1nnn(CC2CC2)c1Br', 'OCc1ncnc(OC2CCCCCC2)c1N', 'O=C(c1cn[nH]c1)[C@H]1CC[C@H](C(=O)[O-])C1', 'O=C(c1c[nH]cn1)O[C@H]1CC(=O)N(C2CC2)C1', 'Nc1nncn1CCN1C[C@@H](C)OCC1(C)C', 'ClCc1nonc1N[C@@H]1CC[C@H]([NH3+])C1', 'ClCc1nnnn1[C@@H]1CCC[C@@H](C2CC2)C1', 'O=[N+]([O-])c1cc(Cl)c(CN2CCO[C@@H](CO)[C@@H]2C[NH+]2CCCC2)cc1', 'Clc1ccc(C[NH2+]c2nc(C3CC3)n(C3CCCC3)n2)cc1', 'O=C(/C=C/c1ccn[nH]1)O[C@@H]1CCC[C@H](C(F)(F)F)C1', 'ClCc1ccn(C2CCCC2)n1', 'Clc1ccc(Cc2c[nH]nc2[C@H]2CC=CC2)cc1', 'O=C([O-])CSc1ccccc1C(=O)Nc1ccc(OC2CCCC2)cc1', 'Clc1ccc(Br)cc1NCC[C@H](O)c1ccccc1F', 'Clc1ccc(-c2csc3[nH]c(COCN4CCOCCC4)nc23)nc1', 'CCOc1ccc2[nH]c3ccc(CC(=O)N4CCN(C(C)=O)CC4)cc3c2c1', 'Clc1ccc(-c2cc(Br)nc(N3CCC(CN4CCCC4)CC3)n2)cc1', 'Clc1ccc(-c2nnnn2C2(C(=O)[O-])CCCCC2)cc1', 'Clc1ccccc1C(=O)OC[C@@]1(O)CCSC1', 'COc1ncc(CN2CCN(C(=O)c3cnn(C(C)(C)C)c3)CC2)cc1Br', 'O=[N+]([O-])c1cc(OCCc2ccccc2)ccc1N1CCCCC1', 'Clc1cccc(SC[C@@H]2CCC[NH2+]2)c1', 'OCc1nccnc1C[C@@H](C[C@@H]1CCOC1)NC(=O)c1cccnc1', 'CCn1ncnc1C[C@H](NC(=O)c1cnn(-c2ccc(C)cc2)c1)C(C)C', 'ClC[C@@H]1C[C@H](O)C[NH+]1Cc1nc2ccccc2c2ccccc12', 'ClCc1ccc2c(c1)CCC[C@@H]2[NH2+][C@@H]1CCSc2ccccc21', 'COCc1cc(NC(=O)N[C@H]2CC(=O)N(C3CCCCC3)C2)n(C)n1', 'CNC(=O)c1cc([N+](=O)[O-])c(N2CCCC2)c2c1cnn2C[C@@H](C)CC', 'C=CCn1c(=O)c(C)nnc1SCCC(=O)Nc1ccc(C#N)cc1', 'O=C(CNCc1nccs1)N1CCC(Oc2ccc(F)cc2F)CC1', 'Clc1ccccc1Nc1nccn1Cc1ccccc1', 'Nc1ncccc1Oc1ccccc1OCc1ccccc1Cl', 'Clc1ccccc1Nc1cnc(NCc2cccnc2)nc1', 'CNc1ccc(Cl)c(C)c1C(=O)N(C)[C@@]1(C)CCCC[C@@H]1C', 'O=C(c1cnccn1)N(Cc1cccc(Cl)c1)c1nc2ccc(F)cc2s1', 'Oc1cc(F)cc(C[NH2+][C@@H]2CC[NH+](CC3CC3)C2)n1', 'Clc1ccc(-c2csc3sccc23)cc1', 'Oc1cc(Br)ccc1Nc1cnccc1C[NH+]1CCSCC1', 'Oc1cc(-n2nnnc2COCC2CC2)cc(OC2CCCCC2)c(C(F)(F)F)c1=O', 'Clc1nnc(COc2ccccc2)n1Cc1ccc2c(c1)OCCO2', 'Clc1ccccc1Oc1cc(OC[C@H]2CCCO2)nc(-c2ccco2)n1', 'Clc1ccccc1-c1nc(CN2CCN(Cc3ccnc(F)c3)CC2)c[nH]1', 'Clc1ccccc1CNc1cnn(-c2cc3ccccc3nc2Cl)c1', 'Oc1cc(F)nc(S[C@@H]2CCOC3(CCOCC3)C2)c1', 'C=C(C)C[NH+]1CCCC[C@@H]1c1nc(C2CC2)no1', 'Oc1cccc(F)c1NC(=O)c1c[nH]c2ccccc12', 'Clc1ccccc1Nc1nnc(Cc2ccccn2)o1', 'Clc1ccccc1CNc1ccc(-n2cccn2)nc1', 'Oc1ccc(F)cc1CN1CCN(c2[nH+]ccc3ccccc23)CC1', 'CNS(=O)(=O)c1ccc(C(=O)NC[C@@H](c2ccc(Cl)c(Cl)c2)[NH+](C)C)cc1', 'CC(C)c1cc(F)ccc1S(=O)(=O)Nc1ccc(Cl)c(Cl)c1', 'Oc1ccc(/C=N/Nc2nnnn2-c2ccccc2)cc1', 'Oc1ccc([N+](=O)[O-])c(C[NH2+]Cc2ccccc2Br)c1', 'ClCc1nonc1[C@@H]1CCCN(c2ncccn2)C1', 'Clc1ccccc1OCC1[NH2+]CC1', 'Clc1ccc(Br)cc1N(CCS(=O)(=O)c1ccc(Cl)c(Cl)c1)[C@H]1CCS1(=O)[O-]', 'Clc1nncn1C[C@@H](OCc1ccccc1)C1[C@@H](O)CO1', 'O=C(Cc1nn(-c2ccccc2)c2c1CCC2)c1ccccc1', 'O=C(CNc1cnnc2cc(Cl)c(Cl)cc12)NCc1ccco1', 'O[C@@H]([C@@H](O)c1ccccc1)CN1CCCCC1=O', 'Clc1ccc(-c2cccnc2N=C2CCCCC2)cc1', 'O=C(/N=C\\NC(=S)Nc1cccc2c1CCCC2)Nc1cc(Cl)ccc1Cl', 'Clc1ccccc1[C@H](Cl)C1CCCC1', 'N=Cc1nonc1[C@H]1NC(=S)N=C1N1CCN(c2ccccc2)CC1', 'Cc1nonc1CC(=O)N1CCCC[C@@H]1C(=O)[O-]', 'O=C(/C(CO)(c1ccccc1)c1cc2ccccc2o1)c1ccccc1', 'ClCc1nnc(-c2ccco2)n1-c1cccc2ccccc12', 'ClC[C@H](CC1CCN(C(=O)c2cccnc2)CC1)n1cccn1', 'Clc1ccc(Br)cc1/N=N\\S(=O)(=O)CCN1CCCC1=O', 'O=C(c1c[nH]nc1Nc1nsnc1C(F)(F)F)N1CCCN(C(=O)C(F)(F)F)CC1', 'Clc1ccc(-c2cc[nH]n2)nc1NC[C@H](O)COc1cccnc1', 'Clc1cccc([C@@H]2C[C@@H]2C[NH+]2CCC(n3cc[nH+]c3)CC2)c1', 'O=C(/C(Cc1ccc[nH]1)c1cccc(F)c1)c1ccc(Cl)cc1', 'O=C(CN(c1cncnc1)S(=O)(=O)c1ccc(Br)cc1)N1CCCCN1', 'Clc1ccc(-c2nnnn2-c2ccccc2)cc1', 'CNc1cccc(OCC(=O)N2CCCC2)c1', 'O=C(/C=C1/NC=CCC1)Nc1ccc(Cl)cc1Cc1ccccc1', 'O=[N+]([O-])c1ccc2nccnc2c1C[NH2+]C[C@H]1CCCO1', 'O=[N+]([O-])c1ccnc2c1CN(C(=O)CCCc1cccs1)C[C@@H]2O', 'NS(=O)(=O)N[C@H]1CC[NH+](Cc2ccc(F)cc2)C1', 'NS(=O)(=O)/N=C(\\NCc1ccc(F)cc1)C1CCCCC1', 'Clc1cc(N2CCO[C@H](CO)C2)cc(Cl)c1Cl', 'Clc1ccccc1NS(=O)(=O)NC[C@@]1(O)CCCO1', 'O=C(COCCN=C(c1ccccc1)c1ccccc1)Nc1nccs1', 'Clc1cc(Br)cnc1O[C@@H]1CC[C@@H]([NH2+]C2CC2)C1', 'Clc1ccccc1-c1cscc1F\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1-c1cscc1F\n\n\n\n\n\n\n\n', 'Clc1ccccc1-c1cscc1F\n\n\n\n\n\n\n', 'Clc1ccccc1-c1cscc1F\n\n\n\n\n\n', 'Clc1ccccc1-c1cscc1F\n\n\n\n\n', 'Clc1ccccc1-c1cscc1F\n\n\n\n', 'Clc1ccccc1-c1cscc1F\n\n\n', 'Clc1ccccc1-c1cscc1F\n\n', 'Clc1ccccc1-c1cscc1F\n', 'Clc1ccccc1-c1cscc1F', 'Clc1ccccc1-c1cscc1F', 'Clc1ccccc1-c1cscc1Br', 'OCc1nc([C@H]2CCO[C@H]2c2ccccc2)no1', 'Clc1ccc([C@@H]([C@@H](Cl)c2ncc[nH]2)N2CCOCC2)cc1', 'Oc1ccccc1-c1cccc2cccnc12', 'O=[N+]([O-])c1c(OCc2nccn2Cc2ccccc2)oc2cc(Cl)cc(Cl)c12', 'Oc1cc(COc2ncnc3c2cnn3Cc2ccccc2)on1', 'Clc1cc(CC2(C[NH2+]CC[C@H]3CCCCO3)CCC2)ncn1', 'N#CCSCCCNC(=O)c1cccn1C(F)F', 'O=[N+]([O-])c1ccc(N2CCN(Cc3ccc3c(c3)CCCC3)CC2)nc1', 'Clc1ccsc1[C@@H](Br)c1ccc(Br)cc1', 'Clc1ccccc1Oc1cnc([C@@H]2CCOC3(CCCCC3)C2)nc1N', 'Clc1ccsc1CN1CCN(C(=O)NC2CCCCCC2)CC1', 'OC[C@H]([S@@](=O)C1CCOCC1)N1CCCc2ccccc21', 'Clc1ccccc1[C@H](O)CCCNC[C@H]1CCCC[NH+]1C', 'Oc1ccc(O[C@H]2CCOC3(CCOC3)C2)cn1', 'Clc1cc(/N=C\\c2ccc(OCc3ccccc3)cc2)nc(Cl)c1', 'Clc1ccc(Br)c(SC[C@@H]2OCCCO2)c1', 'Clc1ccc(C(=O)NCCOc2ccccc2)cc1', 'NS(=O)(=O)/N=C(\\NCc1ccccc1)c1ccc(-c2ccc(Cl)cc2)cc1', 'Oc1cc(OCOCc2nc3ccccc3s2)ccc1N1CCOCC1', 'Clc1cccc2nc(N[C@@H]3CCO[C@]4(CCOC4)C3)oc12', 'Clc1ccccc1NS[C@H]1CCCC[C@@H]1C(=O)[O-]', 'O=C([C@H](CSc1ccccc1)NCc1ccccc1)c1ccccc1', 'OCCC(=O)N1CCN(C(=O)c2cccs2)CC1', 'Nc1nonc1C(=O)C[C@H](O)c1ccccc1', 'Oc1cc(Br)cc([C@H](c2ccccc2)N2CCOCC2)c1', 'Clc1ccc(-c2cc(=O)n3nc(-c4ccco4)s3c2)cc1', 'O=C(Cc1nc2ccccc2[nH]1)NNc1ccccc1', 'Clc1ccccc1N1CCN(Cc2cccc(S(=O)(=O)N3CCCCC3)c2)CC1', 'OC[C@@H]1SC(N2CCCC2)=NC(=O)C[C@H]1[C@@H]2CCCN(C[C@H](O)c1ccccc1)CS2', 'Clc1nncnc1[N-]N1CCN(C(=O)C2CCCCC2)CC1', 'Clc1cc(CCN[C@H]2CCCc3ccccc32)ccn1', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(C(=O)N(CCC(F)(F)F)C2CCOCC2)c2ccccc12', 'Clc1ccc(-c2nn(-c3ccc(S(=O)(=O)C4CCCCC4)cc3)o2)cc1', 'Clc1ccccc1C[NH+]1CCN(c2c(Cl)cccc2[N+](=O)[O-])CC1', 'OC1CCC([NH2+]Cc2ccco2)CC1', 'NS(=O)(=O)c1ccc(CNS(=O)(=O)c2cc(Br)ccc2Cl)cc1', 'O=C(/C=C\\N1CCC(OC2CCCCC2)CC1)c1ccccc1F', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c(=O)c12', 'O=C(/C=C\\CS(=O)(=O)N1CCCSCC1)Nc1cc(Br)ccc1O', 'Clc1ccccc1OCc1ccnn1C1CC1', 'Clc1ccccc1Oc1cnc(/N=C/c2ccccc2Cl)cc1', 'Clc1ccccc1CSC[C@H](O)COc1ccc(F)cc1', 'O=C(CCc1ccncc1)N/N=C/c1ccc(-c2ccccc2F)o1', 'O=C(c1cc(Br)c[nH]c1=O)[C@@H]1CCC[NH+](Cc2ccc(Cl)cc2)C1', 'Clc1cccc([C@@H]2Nc3cc(OCc4ccco4)cc(Br)c3[C@@H]2C(=O)[O-])c1', 'Clc1cccc(S[C@H](C(=O)[O-])C2CCOCC2)c1', 'O=C(/C=C1/c2ccccc2S1)[C@]1(Nc2ccccc2)SC2=c3ccccc3=[NH+][C@@H]21', 'Oc1cc(O)cc2c1C[C@H](CNCc1n[nH]c(C3CC3)n1)C2', 'Clc1ccc(-c2ccoc2C2CC(CO)C2)cn1', 'O=[N+]([O-])c1cc(OCCC(=O)NC2CC[NH+](Cc3ccccc3)CC2)ccc1Cl', 'N#CC1=C([O-])CC2=C(C(=O)N(Cc3cccc4ccccc34)C2)[C@@H]1c1ccccc1Cl', 'Clc1ccccc1Nc1cc(NS(=O)(=O)c2ccc(F)cc2)c(Cl)cc1Cl', 'NS(=O)(=O)C[C@@H]([NH+]1CCc2ccccc2C1)c1ccccc1', 'O=[N+]([O-])c1ccn(CC[C@@H]2CCCC[NH2+]2)c1', 'C=C[C@@H](C)C(=O)NCCNC(=O)c1n[nH]c(C)c1C', 'Clc1cc(Br)c([O-])c(C(=O)N2CCC(c3nccn3CC3CC3)CC2)c1', 'Clc1ccc([C@@H]([O-])NCc2cc(Cl)ccc2[N+](=O)[O-])cn1', 'O=C(c1cn(Cc2cccs2)nc1-c1ccc(Cl)cc1)N1CCOCC1', 'Cc1c(Br)cc2c(c1)N(C(=O)[C@@H](C)CCS(C)(=O)=O)CC2', 'O[C@@H]1C[C@@](O)(/N=C/c2cccn2Cc2ccc(Cl)cc2Cl)CC1', 'CNC(=O)NCC(=O)Nc1ccc(N(Cc2ccccc2)C(C)C)nc1', 'O=C(c1cn[nH]c1)N(C[C@@H]1CCCO1)c1ccccc1', 'NS(=O)(=O)N(Cc1ccc[nH]1)C[C@@H]1CCCCO1', 'OC[C@@]1(CNCc2cc3ccc(F)cc3n2C2CC2)CCCO1', 'Clc1ccccc1CN(Cn1ncc2ccccc21)C1CC1', 'Oc1ccccc1[C@@H]1N[C@@H](c2cc(OCc3ccccc3)ncn2)CCO1', 'O=C(Cc1nn[nH]c1C(=O)c1ccc2c(c1)CCCC2)c1ccccc1', 'Clc1ccc(-c2n[nH]c3[nH]c4ccccc4c23)cc1', 'Clc1nnnn1CCOc1ccc(Br)cc1', 'Oc1cc(Oc2ccc(CS(=O)(=O)Cc3ccc(Cl)cc3)cc2)ccc1F', 'Clc1ccccc1O/N=C(/[O-])N(CC(F)(F)F)C[C@@H]1CCCO1', 'Oc1cc(Cn2cnc3ccccc32)c2[nH]ccc2n1', 'O=[N+]([O-])c1c(Cl)nsc1C(=O)NC1(CC2(O)CCCC2)CC1', 'Oc1ccccc1O[C@@H]1CCC[C@@H](C(=O)[O-])C1', 'OC[C@@H]1c2ccccc2C(=O)N1C[C@@H]1CCCO1', 'N[C@@H](O)C[NH+]1CCC([C@H]2CCOC2)c2ccccc21', 'Clc1cccc2[nH]ncc1n1/C(=C/C(=O)N(Cc3ccccc3)c3ccccn3)c([O-])c12', 'Clc1ccccc1NS(=O)(=O)c1ccccc1N[C@H]1CCSc2ccccc21', 'O=C(c1c[nH]cn1)[C@H]1CCS(=O)(=O)C1', 'Clc1ccccc1OCc1ccncc1', 'Nc1[nH]ncc1C[NH+]1CCC(Oc2ccccc2F)CC1', 'ClCc1ccn(Cc2nc(N3CCOCC3)nc[nH+]2)c1', 'O=C(/N=C/NCc1ccccc1)c1ccc(F)cn1', 'Clc1ccc(-c2ccccc2CC2(O)COC2)cc1', 'N[C@@H]1[C@H](OCc2ncc[nH]2)OC[C@@H](OCc2ccccc2)C1', 'Clc1nnnn1c1c([N+](=O)[O-])cccc1OCC(F)F', 'O[C@@H](Sc1cnn(CCO)c1)C(F)(F)Cl', 'Oc1ccc(Oc2ncccc2C2(C[NH2+]C(F)(F)F)CC2)cc1', 'Clc1cc(/C=C/NC2CC[NH+](Cc3ccccc3)CC2)ccc1F', 'O=C(c1c[nH]cn1)[C@H]1[C@@H](C2=CC=CC=SC2)N=NN1c1ccccc1Cl', 'Clc1ccc(C2S[C@@H](CO[C@H]3OCc4ccccc4O3)C2)cc1', 'Clc1ccccc1-c1cnccn1', 'Clc1ccccc1-c1cnccn1', 'Clc1ccc(-c2ccccc2)nn1', 'Oc1cccc(CC(=O)N2CCC[C@H]2[C@@H]2CCCC2=O)c1', 'Clc1ccccc1COCc1ccc(OC[C@H]2CCCO2)cc1Cl', 'Clc1ccc([C@@H](N[C@@H](C#N)Cc2nc3ccccc3[nH]2)C2CC2)cc1', 'O=C(CNCc1noc2ncc(-c3ccccc3)cc12)Nc1ccccc1F', 'Oc1ccc(O[C@@H](Cn2cccn2)C(=O)c2c[nH]c3ccccc23)cc1F', 'Clc1ccc(Br)c(/N=N/Sc2nnc(N3CCOCC3)s2)c1', 'O[C@@H]1Cc2ccccc2CN1C(=O)NCC[C@@H](O)c1ccccc1', 'N=Cc1nc(-c2ccccc2)nc2ccccc12', 'O=C(CN(CCc1cccnc1)C1CC1)Nc1ccccc1C(F)(F)F', 'ClCc1cn(C2CCCCCC2)nn1', 'Clc1ccc(CC(=O)N[C@@H]2CCc3ccccc32)cn1', 'Clc1ccc(-c2ccccc2)c(N/C=C2\\C(=O)N(c3ccc(Cl)cc3)C2=O)c1', 'ClC[C@H](CO)Nc1ccc(C#Cc2ccccc2)cc1', 'Clc1ccc(Br)c(Cl)c1CN1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1', 'C[C@@H](c1ccccc1)N(C)C(=O)Cc1c[nH]c2ncccc12', 'Clc1ccc(-c2nnnn2[C@@H]2CCC[NH+](Cc3cc(F)ccc3F)C2)cc1', 'Clc1cc(/C=C(\\Cl)CO)ccc1OCOc1cccc(Cl)c1', 'O=C(Cc1cnc2ccccc2n1)Nc1ccc(N2CCCC2=O)c(Cl)c1', 'Clc1ccccc1CN(Cc1ccccc1)C(F)(F)F', 'OC[C@H]1CN(c2nc(-c3ccc(Cl)c(Cl)c3)no2)CCO1', 'ClC[C@H](C[NH+]1CCc2cc(Cl)ccc2N1C[C@@H]1CCCO1)N1CCCC1', 'ClC(F)(F)COCO[C@H]1CCOc2ccccc21', 'O[C@@H](c1cc(OC(F)F)cc(Br)c1)[C@@]1([NH+]2CCCCC2)CCOCC1', 'O=[N+]([O-])c1cc(OC/C(NC[C@H]2CCCO2)c2ccccc2)ccc1Cl', 'O=C(COc1cc(F)ccc1Cl)NN[C@H]1CCOc2ccccc21', 'Clc1ccccc1[C@H](CO)NCc1ccc([N+](=O)[O-])c([N+](=O)[O-])c1', 'Clc1ccccc1Oc1cc(NC2(C(=O)[O-])CCOCC2)ccc1F', 'Clc1cc(Cl)c(C(=O)COC(=O)CCCc2ccco2)cc1F', 'Clc1cc(C(=O)COC[C@H]2COc3ccccc3O2)c(F)cc1F', 'COC[C@@H](C)NC(=O)c1nn(-c2ccccc2)nc1C', 'Clc1ccccc1Oc1cc2c(cc1OCCCOCCO)C(=O)CC2', 'OCc1nccn1CCCOc1cc(Cl)ccc1Cl', 'O=C(COc1cc([N+](=O)[O-])ccc1NC(=O)N1CCN(c2cc[nH+]cc2)CC1)c1ccc(Cl)cc1', 'Oc1ccc(/C=C/N=C(\\[O-])COc2ccccc2)cc1', 'CNS(=O)(=O)c1c(C)c(C)nn1CC(=O)N1CCC[C@@H](C)CC1', 'NS(=O)(=O)Nc1ccc(C(=O)OC(C(C)C)n2ccnc2)cc1', 'Clc1ccc(Cc2nccnc2C(=O)NNC(=O)C2CC2)cc1', 'ClCc1csc(C[NH2+]C2CCCC2)n1', 'OCc1n[nH]cc1[C@@H]1CCC[NH2+]C1', 'N[C@@H]1[C@H]2CCCC[C@H]2[NH+]1Cc1ccncc1', 'Cc1c([C@@H]2CCC[NH2+][C@@H]2C)cnn1C', 'O=[N+]([O-])c1c(Cl)n2nnc(/N=C/c3ccccc3)c2nc1O', 'CC(=O)N1CCC(CCC(=O)NNC(=O)c2cn(C)nc2-c2ccccc2)CC1', 'Clc1cccc2ncnc(NCC[NH+]3CCC(O)CC3)c12', 'Clc1ccc(C2CC[NH+](C[C@@H](O)c3cnccn3)CC2)cc1', 'O=[N+]([O-])c1c(O[C@@H](CO)c2ccncc2)cccc1[N+](=O)[O-]', 'Clc1ccccc1NS[C@H]1NC(=O)c2cc(C(=O)NC3CCCCCC3)ccc21', 'O=[N+]([O-])c1ccn(Cc2ccccc2)c1', 'O=C([C@H](Sc1nccc(Cl)c1F)N1CCCC1)c1ccc2c(c1)OCCO2', 'Clc1ccc(Cc2noc(/N=C/C=C/c3ccccc3)n2)cc1Cl', 'Clc1cccc2c1C[C@@H](Cc1cccnc1)C2', 'Clc1ccccc1CNC[C@H]1CCC[C@H](CNS(=O)(=O)C2CC2)C1', 'CNS(=O)(=O)C[C@H](C)ONC(=O)c1ccc2c(c1)CCO2', 'Clc1ccc(C[NH2+]c2cccnc2)cc1', 'CCO[P@](=S)(C(=O)[O-])c1ccccc1', 'Clc1ccccc1OC1CC[NH+](Cc2cccc(Cl)c2)CC1', 'O=C(CCS)C1=Cc2ccccc2OC1', 'Clc1ccccc1OC1(NC(=O)c2ccc(Cl)cc2F)CC1', 'Clc1cc(Cl)ccc1-c1ccccc1CCNC(=O)N1CCOCC1', 'Clc1ccccc1Oc1cncc2ccccc12', 'Clc1ccccc1Oc1cncc2ccccc12', 'Clc1ccccc1Nc1ccnc2ccccc12', 'O=C(CNCc1ncoc1Cc1ccncc1)Nc1ccc(Cl)cn1', 'Nc1nc2ccccc2n1CCOCc1ccccn1', 'Nc1cnnn1-c1ccc(F)cc1C(=O)N[C@H](c1ccccc1)C1CC1', 'OC1(SSCc2cccc3cc(F)ccc23)CCCCC1', 'C=CC#CCCNC(=O)Nc1ccc(-n2ncn(C)c2=O)cc1', 'O=C([O-])/C(=C\\Cc1ccccc1)n1nnc2ccccc21', 'O=C(CCS)Nc1ccccc1C(=O)Nc1ccccc1C(F)(F)F', 'Clc1cc(Cl)cn2cc(-c3ccccc3)cc12', 'O=C([C@H](Sc1nnnn1Cc1ccccc1)[C@H]1CCCO1)N1C[C@H](CO)CCO1', 'Clc1ccccc1N1N(CC2CC2)C(=O)N1Cc1cccc(F)c1', 'Clc1ccccc1-c1nnc2n1NCC[C@@H]1C[C@@H]1C2', 'Clc1ccccc1-c1ncc(-c2ccccc2Cl)nc1N1CCCC1', 'Clc1nncc(Oc2ccc(F)cc2)c1N[C@H]1CCCOc2c(Cl)cccc21', 'O=C(c1cc(F)c(F)cc1F)N1CCC([NH2+]CC[NH+](C)Cc2ccccc2)CC1', 'Clc1ccc(C2SCCCS2)cc1', 'Clc1ccc(C2SCCCC2)cc1', 'Oc1cc(COc2nc(-c3ccc(Cl)cc3)nc3c2CCC3)ccn1', 'Clc1ccccc1Cc1c(O)ccc2cccnc12', 'Clc1cc(Cl)c2ccccc2c1C[NH+](Cc1ccccn1)C1CCCCC1', 'Oc1cc(-n2nnnc2CN2CCN(c3ccccc3F)CC2)ccc1NC[C@@H]1CCCO1', 'Oc1ccccc1/C([O-])/N=C/c1ccccc1', 'ClC(=O)N[C@H](Cc1cccs1)C(=O)[O-]', 'COc1ncc(Br)cc1C(=O)N[C@@H]1C[C@@H](C)N=C(C#N)[C@H]1c1ccccc1Cl', 'OCc1nc2ccccc2c(=O)n1-c1sc2c(c1-c1ccccc1)CCCC2', 'Oc1cc(OC[C@@H]2CC[C@@H](C(F)(F)F)N2)c2ccccc2c1C1CCCCCC1', 'O=C(/C=C/c1ccc(Cl)cc1)Nc1ncc[nH]1', 'COc1nn(C)c(CC)c1CNCCNC(=O)c1ncccc1OC(C)C', 'O[C@@H]([C@@H]1CC(C(C)C)=C(C)CS1)c1ccc(Cl)cc1', 'C=CCCC(=O)Nc1ccc2c(SCC(F)F)cc(C)nc2c1', 'O=C(/C=C1/NCCN1C(=O)Cc1ccc(Cl)cc1)Nc1cn[nH]c1', 'Clc1ccc(C(F)F)cc1NC[C@H]1CCCN(c2ccccc2)C1', 'Clc1ccccc1-c1noc(C2CCCCC2)n1', 'Clc1ccccc1Cn1nc2cc(-c3ccccc3)ccn2c1=O', 'O=C(CS(=O)(=O)c1ccc(Cl)cc1)NCCc1ccccc1F', 'Clc1ccccc1OCCNS(=O)(=O)c1ccccc1Br', 'O=C([C@H](c1ccc([O-])c(C(F)(F)F)c1)N1CCCCCC1)N1CCCS1(=O)=O', 'O=C(COCCO)Oc1ccccc1F', 'O=C(/C=C/OC1CCCC1)Nc1ccccc1[N+](=O)[O-]', 'Oc1ccccc1-n1cccn1', 'Clc1ccc(Br)cc1SCCCC1[NH+]=c2ccccc2=[NH+]1', 'Clc1cc(F)c(C[S@](=O)CS(=O)(=O)N2CCCC[C@@H]2CO)cc1', 'Cc1c(Br)onc1CCN(C)C(=O)NCc1ccccc1', 'Clc1nnc([S-])c2c1N=C1C[C@H](Cl)CC1(CC[NH+]1CCOCC1)C2', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n', 'Clc1cccc2cccnc12\n\n', 'Clc1cccc2cccnc12\n', 'Clc1cccc2cccnc12', 'Clc1cccc2cccnc12', 'Clc1cccc2cccnc12', 'Clc1cccc2cccnc12', 'Clc1cccc2cccnc12', 'OC[C@@H]([NH2+]Cc1ccccc1Br)C(F)(F)F', 'O=C(c1ccsc1)c1csc2ccccc12', 'O=C(CC(=O)N1CCOCCN(S(=O)(=O)c2ccc(Cl)cc2)C1)N1CCOCC1', 'Clc1ccccc1OCCOCCOC[C@@H]1CCCO1', 'Oc1cc(O)ccc1S(=O)(=O)N1CCC[NH+](Cc2ccc(F)cc2)CC1', 'Oc1ccccc1CCN1CC[NH+](Cc2cc(F)ccc2F)CC1', 'ClC[C@@H]1C=C[C@@H](c2ccccn2)C1', 'Clc1ccccc1N1NC2=NC=C2OC(=S)N1C1CCCCCC1', 'Oc1cc(F)ncc1C[NH2+]C[C@@H]1CN(Cc2ccccc2F)CCO1', 'O=C(/C=C\\c1ccc([S@](=O)C(F)F)cc1)NCCNC(=O)c1ccccc1', 'O=C(/C(Cc1ccccc1)c1cccc(F)c1)c1ccccc1', 'Clc1ccccc1CCC[NH+]1CCC[C@@H]1c1cccs1', 'Clc1ccccc1NC1C2CCC(C2)C1', 'Clc1nnc([N+](=O)[O-])c(OC2CCCCC2)n1', 'O=C(Cc1n[nH]c(=O)[nH]1)Nc1ccc2c(c1)oc(=O)n2Cc1ccccc1', 'Clc1nncnc1[N-][C@@H]1CCC[NH+](CC(F)(F)F)C1', 'Clc1cccc2c1N(Cc1nccn1Cc1ccccc1Cl)CCCC2', 'O=C(c1ccccn1)N1CCO[C@H](c2ccoc2)C1', 'Clc1ccsc1N/C(=C/c1ccc(Cl)cc1)NCc1ccccc1', 'O=C(Cc1coc(-c2ccccc2Cl)n1)NC1COC1', 'Nc1[nH]ncc1COC(=O)[C@H]1C[C@H]1c1ccc(Cl)cc1', 'Clc1ccccc1N1N=C(C(F)F)[C@@H](c2cccc(F)c2)C(=O)N1c1ccccc1Cl', 'C=C[C@@H](NC(=O)c1ccc(F)cc1)c1ccccc1OC', 'Oc1cc(CCn2nc(-c3cc(F)ccc3F)nn2)ccc1Cl', 'OCS1(=O)CCN(C(=O)COc2ccccc2)CC1', 'CC(C)CC[C@H](C)[NH2+]Cc1cccc(O[C@@H](C)C(=O)OC(C)(C)C)c1', 'O=C(Cc1non1C1CC1)NCc1ccnc(-n2cccn2)c1', 'O=C(/N=C\\NC1=NC=NS1)c1csc(-c2ccc(F)cc2)n1', 'Clc1ccccc1COCCNS(=O)(=O)c1ccccc1', 'Clc1nnnc(NN[C@H]2C[C@@H]3COC[C@@H]3C[C@@H]2C2CC2)c1', 'Oc1ccc(/C=N/Nc2nccc(N3CCCC3)n2)cc1Cl', 'Clc1ccc(-c2nnnn2CCNc2ccc(F)cc2Cl)cc1', 'Clc1ccccc1-c1csc(Cc2ccc(F)cc2)n1', 'Clc1ccc(C2CC2)cc1NCc1nc(C2CC2)no1', 'Cc1sn(-c2ccccc2)c1C(=O)N[C@H](c1cccc(Cl)c1)c1ccccc1', 'Clc1ccccc1Oc1cnccc1Oc1ccc(OC(F)(F)F)cc1', 'ClCc1ccnnc1SCC(=O)NC[C@@H]1CCCO1', 'CN[C@H](CN1CC[NH2+]CC1)Cn1cc[nH+]c1C', 'COc1ncnc(Br)c1C(=O)N1CCC(OC)CC1', 'Clc1ccc(Br)cc1/N=N\\n1ccc2ccccc21', 'Clc1ccc(-c2cc[nH]n2)c(C[NH2+]CCc2ccccc2)c1', 'Clc1ccc(-c2csc3[nH]c(Nc4ccccc4)cc23)cc1', 'Clc1ccc(C[NH+](C[C@@H]2CCOC2)C[C@H]2CCCO2)cc1', 'Clc1ccccc1-c1cscc1-c1ccccc1', 'Clc1ccc(-c2csc3ccccc23)cc1', 'Oc1ccc(F)c(F)c1CNc1cccc2c1CCCC2', 'Clc1ccccc1NC(=O)[C@@H](CO)CCl', 'N#C[C@@H]([C@@H]1CCCON1c1ccccc1)c1ccc(-n2nnnc2CS)cc1', 'Oc1cc(CCCn2ccc(CO)n2)ccc1CO', 'O=[N+]([O-])c1ccn(CC2=CS(=O)(=O)N(c3ccccc3)c3ccccc32)c1', 'ClCc1cn(CCO)nn1', 'Nc1[nH]ncc1SCC(=O)N1C[C@@H](O)C[C@H]1c1ccc(Cl)cc1', 'Oc1ccc(/C=C/N(CC(F)F)c2ccccc2F)cc1', 'O=[N+]([O-])c1c(OCc2nsc(-c3cccs3)n2)oc2ccccc12', 'Nc1c(Cl)cccc1S(=O)(=O)N[C@@H](c1ccccc1)c1cccc2nc[nH]c12', 'Clc1ccsc1/N=C/c1cccc(-c2c(F)cccc2F)c1', 'O=C(CC[NH+]1CCC(O)(c2ccccc2)CC1)Nc1cccc2ncccc12', 'C=CCOc1ccc(/C=N/NC(=O)Cc2ccc(C)cc2)cc1', 'Oc1cccc([C@H]([NH2+]C2CCCC2)C2CC2)c1', 'ClC[C@H](O)[C@@]1(Cc2ccccn2)N2Cc3ccccc3[C@H](O)[C@H]12', 'Oc1ccc(F)cc1OCCN1CCN(c2cc[nH+]cc2)CC1', 'O=C(Cc1cnco1)Nc1cccc2c1CCCC2', 'Cc1nonc1N[C@H](C)C(=O)NCc1ccccc1Br', 'Nc1[nH]ncc1NS(=O)(=O)CC1CCCC1', 'Clc1ccc(Br)c(-c2ccco2)c1N', 'OC[C@H](CO)Nc1ncnc2c1c(=O)n(C)c1ccccc12', 'O=C(COCc1ccncc1)NCCN1C[C@@H]2CC[C@@H]1CN(S(=O)(=O)c1cccs1)CC2', 'O=[N+]([O-])c1ccn(CC[C@H]2CCC[C@@H]2O)c1', 'Clc1ccc(-c2nn(-c3ccccc3)cc2F)cc1', 'Clc1cc(Cl)ncc1/C=C/C(=O)C(c1ccccc1)c1ccccc1', 'N=C1\\[C@@H](SCCn2c(N)nc3ccccc32)[C@H]2C=CCCCN12', 'Oc1ccccc1[C@@H]1CC[NH+]2CCCC[C@@H]12', 'Clc1ccccc1NCc1ccnc(OC2CCC2)c1', 'Clc1ccccc1-c1nc(C[NH+](Cc2ccco2)Cc2ccco2)co1', 'O=C([C@H](c1cccs1)N(c1cccc(Cl)c1)N1CCCC1)c1cccc(F)c1', 'Oc1cc(-n2cnc(NNC(=O)C3CCCC3)c2Cl)cc2c1CCC(=O)N2', 'Oc1cc(-n2nnc3c(F)cccc32)c([N+](=O)[O-])c2ccccc21', 'Clc1ccccc1NS(=O)(=O)Nc1ccc(CC2CC2)cc1', 'Oc1ccccc1OC1CC(C(F)(F)F)O[C@@H]1c1ccccc1', 'O=C(COc1ccc2ccccc2c1)NCC#CCOc1ccc(F)cc1', 'COc1nnc(SCc2ccc(C(=O)[O-])cc2)n1CCOC(C)C', 'Clc1ccc(-c2cc(-n3ccnc3)cn3cnnc23)cc1', 'Clc1ccc(C2SCCC2)cc1Cl', 'O=[N+]([O-])c1c(-c2c[nH]c3ccccc23)nc2ccccn12', 'Oc1cc(-n2cn/cc2)ccc1N[C@@H]1CCC[C@H](C2CC2)C1', 'Clc1ccccc1-c1csc(C[NH2+]C2CCCCCC2)n1', 'O=C(C1CCC1)N1CCN(C(=O)c2cnc(CN3CCOCC3)s2)CC1', 'O=C(C1CCC(Nc2nccs2)CC1)N1CCC(n2ccccc2=O)CC1', 'Clc1cc(F)cc2c1CCN(C(=O)Cc1ccc(OC(F)F)cc1)CC2', 'Oc1cccc(Br)c1OC[C@H](O)CBr', 'Clc1ccc(Br)cc1C[NH+]1CCC[C@H]1COc1ccc(Cl)cc1', 'Oc1cc(F)c(C(=O)NCC2(c3ccccc3)CCOCC2)cc1Cl', 'Oc1cc(COC[C@@H]2COc3ccccc3O2)c(Cl)cc1Cl', 'Clc1ccc(Br)cc1[C@@H]([NH3+])COOCc1cccc(F)c1', 'O=[N+]([O-])c1ccncc1/C=N/N=C[C@H]1C=C[C@@H](O)C1', 'Clc1cccc(/C(CC(=O)c2ccc(OCc3cnn(C)c3)cc2)N2CCCC2)c1', 'Clc1cccc(-c2cn(CC(CC[NH3+])C3CCN(c4cc[nH+]cc4)C3)cn2)c1', 'O=[N+]([O-])c1cc(C2CCCCC2)n2ncc(Cl)c2c1', 'O=C(c1c[nH]c2ncn(Cc3ccccc3)c12)N1CCCCCC1', 'Clc1ccc(-c2n[nH]cc2C[NH+]2CCCN(Cc3ccc(F)cc3F)CC2)cc1', 'Clc1ccccc1OC1(Oc2cnc(-c3cscn3)s2)CCOCC1', 'Clc1ccccc1OCc1ccc(Br)c(N2CCOCC2)c1', 'Oc1cc(Cn2ncc(Cl)c2[N+](=O)[O-])cnc1Cc1ccccc1', 'Clc1cc(F)cc(CN2CC[NH+](CCOc3ccccc3)CC2)c1', 'Clc1nncc(C[NH+]2CCC[C@@H]2c2ncc[nH]2)n1', 'O=[N+]([O-])c1c(F)ccc2c1N(Cc1ccc(Br)cc1)CC2', 'OCS(=O)(=O)c1ccc(F)cc1Oc1ccc(C(F)(F)F)cc1', 'O=C(Cc1nn[nH]n1)Nc1ccc(C(=O)N2C[C@@H](C)O[C@H](CCc3ccccc3)C2)cc1', 'CCn1c(=S)sc2cc(Br)ccc21', 'Clc1cccc(Br)c1NC[C@]1(O)CCSC1', 'Oc1cccc(CC[C@H]2CCCC[NH2+]2)c1', 'C=CCc1cccc(NC(=O)N2CCC(CNC(=O)C(C)C)CC2)c1', 'Oc1ccc(F)cc1-n1c(=S)[nH]c2cc(Cl)ccc21', 'O=C(c1c[nH]c(=O)[nH]c1=O)N1CCN(c2ccccc2O)CC1', 'O=C(C[NH+]1CCN(Cc2cccnc2)C1)Nc1cnccn1', 'CCNC(=O)[C@H]1C[C@H](c2ccccc2F)N(C(=O)c2ccc(F)c(F)c2F)C1', 'Clc1ccc(-c2cc(O[C@H]3CCCOC3)nc(-c3ccc(Cl)cc3)n2)cc1', 'Clc1nnnn1c1c(Br)cccc1N1CCCCC1', 'O=C(Cc1nnnn1-c1ccc(Br)cc1)Nc1ccc2c(c1)OCCO2', 'C=CCNC(=O)[C@H]1CCC[NH+](Cc2c[nH]nc2-c2ccc(C)o2)C1', 'Clc1ccccc1CSCc1cccc(Br)c1', 'Nc1nnc(Cc2cccs2)n1-c1ccccc1', 'Clc1ccc(COCO[C@@H]2CCO[C@]2(F)c2ccc(F)cc2)cc1', 'Clc1ccc(Br)cc1S[C@H]1CCCc2ccccc12', 'Clc1ccc([C@@H](Br)[C@H]2CSCCO2)cc1', 'ClCc1noc2c1C[C@@H](CO)C2', 'CN(c1ccn(Cc2ccccc2)n1)C1CCCC1', 'O=C(CS(=O)(=O)c1c[nH]c2ccccc12)NNC(=O)/C=C/c1ccco1', 'Clc1cccc(F)c1C[NH+]1CC[C@H](Nc2ccccc2)C1', 'O=C(/C=C1CSCCS1)Nc1ccccc1Cc1c[nH]c2ccccc12', 'Clc1ccccc1NC12CC3CC(CC(C3)C1)C2', 'Clc1ccccc1C[NH+]1CCc2ccccc2[C@H]1C(=O)Nc1cccc(Cl)c1', 'Oc1cc(Br)c([C@H]2CCCN2CC(=O)NCc2ccco2)cc1F', 'O=[N+]([O-])c1cc(Br)ccc1Oc1ccccc1C[NH2+]C[C@@H]1CCCO1', 'Oc1ccccc1/N=c1\\scc(N2CCOCC2)n1-c1ccccc1', 'Clc1ccc(-c2n[nH]cc2CN2CCN(Cc3cccs3)CC2)cc1', 'COc1nnn(-c2ccc(OC)cc2)c1C(C)C', 'Oc1ccccc1-c1c(C(=O)N(Cc2ccccc2)Cc2ccccc2)csc1C#N', 'Clc1ccc(C(=O)c2ccccc2)cc1', 'O=C(c1c[nH]c2ncnc(NC3CC3)c12)N1CCOCC1', 'O=[N+]([O-])c1ccncc1-n1cc([O-])c(NC2CC2)c1', 'Clc1cccc2cc(C[NH2+][C@H](CO)c3ccccc3)c(=O)[nH]c12', 'Nc1[nH]nc(N/N=c2/[nH]c3ccccc3n2Cc2cccs2)c1C(F)(F)F', 'O=[N+]([O-])c1ccncc1Oc1ccc(N2CCOCC2)nc1', 'O=[N+]([O-])c1ccncc1N[C@H]1CCN(c2ccccc2)C1=O', 'NS(=O)(=O)c1cc(NC(=O)COc2ccc(Br)cc2)ccc1O', 'ClCc1nonc1Nc1ccc2c(c1)CCCC2', 'Clc1nnn(C[C@@H](Br)CO)c1-c1ccc(Cl)cc1F', 'Oc1ccc(/N=c2/[nH]cccc2CCNC(=O)c2cccnc2)cc1', 'Nc1c(CN(C)C(=O)[C@H]2CCO[C@H]2C)cnn1-c1ccccc1', 'C[C@@H](NC(=O)Nc1ccccc1)[C@@H]1CCCN(c2ncccn2)C1', 'Clc1cccc([S@@]c2c[nH]nc2C[NH+](CC2CCCC2)C2CC2)c1', 'CN(c1cc(Cl)cc(N)n1)C1CC[NH+]([C@@H](C)C(C)C)CC1', 'Clc1cccc(F)c1Cc1nc(CCCc2ccccc2)no1', 'ClCc1cc(OC[C@H]2CCCO2)nc(OC2CCCCC2)n1', 'O=C(c1c[nH]nc1-c1ccc(Cl)cc1)N(CC1CC1)S(=O)(=O)c1ccc(Cl)cc1', 'ClCc1cc(Br)cnc1N1CC[C@H]([NH+](C)Cc2ccccc2)C1', 'Clc1ccc(CNC[C@@H]2CCCN2C(=O)c2ccc(Cl)cn2)cc1Cl', 'Clc1cccc2[nH]nc(S(=O)(=O)N3CCCC[C@@H]3C[NH+]3CCCC3)c12', 'Clc1cc(Br)cn1N1CC(c2ccc(CO)cc2)C1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1[C@@H]1CCS(=O)(=O)C1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1[C@H]1CCS(=O)(=O)C1', 'Oc1cc(Br)c(N/N=C\\c2ccccc2)cc1Cn1ccnc1', 'Oc1ccc(/C=N/Sc2nnc3c(-c4ccc(F)cc4)nnn3c2[O-])cc1', 'ClCc1ccn(C[C@@H](O)c2cccnc2)c1', 'Nc1nn(-c2ccc(N3CCCCC3)cc2)c2c1CCC2', 'C=CCc1ccccc1OCc1nnc([S@](=O)CC(C)(C)CC)n1C', 'Oc1cc(-n2cc(Cn3cncn3)c(Cl)cc2=O)cc(OCC(F)F)c1', 'Clc1ccccc1-c1cnn2c1NCCC[C@@H]2[NH2+]CCC(C)C', 'Oc1ccc(O[C@@H](C[NH+](C)C)Sc2ccccc2)c(Cl)c1', 'Oc1cccc(CC[C@@H]2CCCCO2)c1', 'Oc1cc(C(F)(F)F)ccc1[C@H]1CCCN(c2cc(-c3ccccc3)ncn2)C1', 'O=C([O-])c1ccc(CN[C@H]2CC(=O)N(c3ccccc3)C2)c(F)c1', 'O=C(Cc1noc(Cc2ccc(Cl)cc2)n1)NCCc1ccccc1Br', 'Oc1ccc(O[C@@H](CNC(=O)C[C@H]2COCCO2)c2ccco2)cc1Cl', 'O=C([C@H](N[C@@H](C(=O)[O-])C1CCCC1)Nc1cccc(Cl)c1)OCCO', 'ClC[C@@H]1CO[C@@H](c2ccccc2)[NH+](CC2CC2)C1', 'Oc1cc(COCOC2[C@@H](N3CCOCC3)CCC2)ccn1', 'CN[C@H]1[C@@H](OCc2ccccc2)N(Cc2ccccc2F)C1=O', 'N[C@@H]1[C@H]([NH2+]C2CCN(c3ccc(C(F)(F)F)cn3)CC2)CCC[NH+]1C', 'O=C(/N=c1/[nH]c(C(F)(F)F)cc(-c2ccncc2)n1)[C@@H]1CCC[NH+](C2CC2)C1', 'O=[N+]([O-])c1c(Cl)nsc1NC[C@H](O)c1ccccc1Cl', 'NS(=O)(=O)NC[C@@]12CCN(c3ccccc3)[C@@H]1Cc1ccccc1-2', 'O=[N+]([O-])c1c(Cl)nn(CNc2ccccc2Br)c1NC(=O)c1cccs1', 'Clc1ccc(Br)cc1-n1ncc2sccc21', 'Clc1ccccc1-c1n[nH]c2c1[C@@H]1CC[C@H]1C[NH2+]C2', 'Clc1ccccc1-c1cnn2c1N=C1CCCC[C@H]1CC2', 'Clc1cccc(CNc2ncnc3sc(-c4ccccc4)cc23)c1', 'O=C([C@H](SC[C@@H]1CCCCO1)c1c[nH]c2ccccc12)C(=O)[O-]', 'Clc1ccccc1OCCn1cccn1', 'ClCc1cccc(C2n3c(cc4ccccc4c3=O)C2)c1', 'O=C([O-])/C=C/[C@]1(CN2C(=O)Cc3ccccc32)CCC1=O', 'O=C(c1c[nH]c2cc(F)ccc12)N1CCC(O[C@@H]2CCS(=O)(=O)C2)CC1', 'O=[N+]([O-])c1c(-c2ncnn2Cc2ccccc2)cc2ccccc2n1', 'O[C@@H](CNc1cnccn1)c1ccc(Cl)cc1', 'OCc1nc(NCc2ccc(Cl)cc2)ncn1', 'Clc1ccccc1OCc1csc2ncc(-c3ccccc3Cl)n12', 'O=C(COc1ccc(F)c(F)c1)N1CCC(Oc2ccc(C(C)=O)cc2)CC1', 'O=C(c1c[nH]nc1N[C@H]1CCSc2ccccc21)N1CC=C(c2ccccc2)CC1', 'O=C([C@H](Sc1noc2ccccc12)C1CCCC1)NC[C@H](O)c1ccccc1', 'ClCc1cn[nH]c1O[C@@H]1CCCC[C@@H]1OC1CCCC1', 'Cc1noc2nc(CN3CCO[C@@H](OCc4ccc(F)cc4)C3)c(C#N)cc12', 'O=C(Cc1conn1)NCCN1CCSC[C@@H]1C[NH+](C)C', 'Oc1cc(-n2nnc(-c3ccc(Cl)cc3)n2)ccc1N1CCCCC1', 'Clc1ncnc(N2CC=C(c3ccc(OC(F)F)cc3F)CC2)c1[N+](=O)[O-]', 'Clc1ccsc1CN[C@@H]1CCC(=O)N(C2CCOCC2)C1', 'Nc1ccc(C(=O)N2CCC3(CC2)c2ccccc2NC(=S)N3C)cc1', 'C=C1[C@H](C)C[C@H](C(=O)O[C@@H](C)c2ccccc2)[C@H]1O', 'O=[N+]([O-])c1ccc2[nH]ncc2c1-c1csc(-c2ccccc2)n1', 'ClCc1ccccc1OCc1cccc2ncccc12', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(Cl)c1Oc1ccccc1Br', 'Clc1cccc2c1OC[C@@H](c1ccccc1F)CN2Cc1ccccc1Cl', 'OC[C@@H]1CN(Cc2cccc([S@@](=O)C(F)F)c2)CCS1', 'Clc1ccsc1[C@H]1N=C(C[NH+]2CCN(c3ccc(Cl)cc3)CC2)C1', 'O=C(c1cccnc1)N1CCC[C@H](c2nccn2Cc2ccccc2)C1', 'OCSc1nnc(N2CCC[C@@H]2[C@@H]2CCCC2)[nH]1', 'C=C[C@@H](Oc1ccccc1OC)c1ccc(F)cc1', 'Clc1ccc(Br)cc1N1/N=C/c2ccccc2N1CCCc1ccccc1', 'N#C[C@H](C(=O)[C@@H]1CCCN(C(=O)Cc2ccccc2Cl)C1)c1ccccc1', 'Clc1ccc(Br)c(F)c1C(=O)NC[C@@H]1CCCc2ccccc21', 'O=C(/C=C1CCCC[C@@H]1C(=O)NCc1coc(-c2cccs2)n1)c1ccccc1Cl', 'Clc1ccc(-c2csc3c(Cl)cccc23)cc1', 'O=[N+]([O-])c1ccn([C@H]2CCCCN2C2CC[NH2+]CC2)n1', 'Nc1n[nH]nc1CNC(=O)C(=O)NC1CC1', 'O=[N+]([O-])c1ccncc1OCCC(F)(F)F', 'O=C(c1ccc[nH]c1=O)N(CCOc1ccccc1F)[C@@H]1CCS(=O)(=O)C1', 'O[C@@H]1c2ccoc2C(=O)N1[C@H]1CCC[C@@H]1CC(=O)Nc1ccc(F)cc1', 'ClC(F)(Cc1ccco1)C(=O)c1ccc(Br)s1', 'Clc1ccc([C@@H](OC[C@H]2CCC[NH2+]C2)[NH+]2CCCC2)cc1', 'Clc1ccc(COC2CC[NH+](C[C@@H]3CCCO3)CC2)cc1', 'Clc1ccc(CNC2CC[NH+](C[C@H]3CCCO3)CC2)cc1', 'Clc1ccccc1-c1nn(C2CC2)c(-n2nc([O-])c3ccccc23)n1', 'O=C([C@H](NSc1ccccc1)[C@H]1CSC(=O)O1)c1ccccc1', 'ClCc1cncc(COc2ccc(C(=O)[O-])cc2F)n1', 'ClC(=S)N(Cc1ccccn1)[C@H]1C[C@@H]2CC[C@@H]1C2', 'Clc1ccc(CN2CCN(C3CC3)CC2)cc1', 'Clc1ccc(CN[C@@H](CO)CC2CCCC2)cc1', 'Clc1ccc(CN[C@@H](CCO)C2CC2)cc1', 'Oc1ccc([C@H]([O-])CCOc2ccc(Cl)cc2)cc1', 'ClC(c1ccccc1Cl)[NH+]1CCC[C@H](n2cccn2)C1', 'Clc1ccccc1NCCCSc1ncnn1C1CC1', 'Clc1ccccc1N1N(Cc2ccccc2)C[C@@H]2CCCC[C@@H]21', 'CNC(=O)c1cc(C(=O)N[C@H](C)c2cc(Cl)ccc2OC)ncn1', 'Clc1ccc(C2SCCS2)cc1', 'Clc1ccc(C2SCCS2)cc1', 'O=C(c1cccs1)N(Cc1ccc[nH]1)c1nc(N2CCOCC2)nc2cccnc12', 'O=C(CS(=O)(=O)[N-]C1CCCCC1)Nc1cccc(C2CCCC2)c1', 'OCc1nnsc1N/N=C/c1cccc(OC2CCCC2)c1', 'Clc1ccccc1OCc1ccn([C@H]2CCS(=O)(=O)C2)[nH+]1', 'O=C(CS(=O)(=O)CCc1ccccc1)NCc1ccccc1F', 'Clc1ccccc1-c1noc([C@H]2CCCN(C(=O)c3ccccc3)C2)c1Cl', 'Clc1cc(CCC(=O)N(Cc2cccs2)C[C@H](O)c2ccccc2)ccc1F', 'Nc1ncc([C@@H](C)[NH2+]C2CC2)s1', 'Clc1cccc(Nc2cn(Cc3ccccc3)nc2C[NH+]2CCCCCC2)c1', 'Clc1cc(F)cc(C[NH+]2CCC[C@H](c3nc(-c4ccco4)[nH]n3)C2)c1', 'Clc1ccc(Cc2nccc(CC[NH+]3CCOCC3)n2)cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1OC[C@@H]1CCCO1', 'O=[N+]([O-])c1c(F)cccc1NC[C@@H]1CCC[NH2+][C@@H]1CO', 'Clc1ccc(-c2n[nH]cc2NCc2csc3ccccc23)cc1Br', 'Clc1ccc(Br)cc1-n1ncc2c1[C@H]2CCCOc1ccccc1', 'Clc1ccc(C(F)(F)F)nc1N1CCCCC1', 'Clc1cccc([S@@]c2nc3c(n2C)CCCC3)c1', 'C[C@@H](Oc1ccc(Cl)cc1)C(=O)N1CCN(S(=O)(=O)c2ccccc2)CC1', 'Clc1cc(F)c(N[C@H]2CC[NH2+]C[C@H]2Cc2ccccc2)cc1', 'Clc1nncn1C1CCN(c2ccc(C(F)(F)F)nc2Cl)CC1', 'N[C@@H](CC(=O)N1CCCC1)c1nc2cc(F)ccc2o1', 'Oc1ccc(/C=N\\NC2=NC=NN2)cc1', 'O=C(CN(CC1CCCC1)c1ccc(F)cc1)NCC1CC1', 'Nc1nnc(SCC(F)(F)F)n1N', 'Oc1ccc(F)c(F)c1CNC[C@@H](O)c1ccc(Cl)cc1', 'O=[N+]([O-])c1c(Cl)nn(C2CC2)c1NCc1csc2ccccc12', 'ClCc1ccn([C@H]2CCO[C@@H]2C(F)(F)F)c1', 'CN[C@H](C#N)c1cc(F)ccc1SCc1ccc(Cl)cc1', 'NS(=O)(=O)/C=C\\c1ccc(-n2cccc(C(=O)Nc3ccccc3)c2=O)cc1', 'O=C(COc1cccc(C(F)(F)F)c1)N1CCC[C@H]1c1ccccn1', 'Clc1ccccc1NS(=O)(=O)c1ccc(Oc2ccccc2)cc1', 'Clc1ccc(-c2cc[nH]n2)cc1CCNC(=O)c1ccccc1', 'Oc1ccccc1C1=NN(c2ccccc2)[C@H](c2ccccc2)C1', 'Clc1cccc(/C(CC(=O)Nc2cc3c(cc2Cl)OCCO3)[NH+]2CCCC2)c1', 'CNS(=O)(=O)CCc1ccc(NC(=O)N2CCc3sccc3C2)cc1', 'O=C(c1c[nH]cn1)N1CCC(Oc2ccc(C(=O)NCc3cccs3)cc2)CC1', 'O=C(c1cc(F)ccc1F)N1CCC[C@@H]1CSc1ncnc2c1cnn2CC(F)(F)F', 'Nc1[nH]nc(C2CC[NH+](Cc3ccccc3C(F)(F)F)CC2)n1', 'ClCc1nc(I)cn1C1CC1', 'Clc1ccc(C(=O)N2CCN(Cc3ccc(Cl)cc3)CC2)cc1', 'COc1nc(C)c(C(=O)O[C@H](C)C(=O)N2CCN(c3ccc(C)cc3)CC2)cn1', 'Clc1ccc(Br)cc1-n1nnnc1NC[C@@H]1CCCCO1', 'Oc1cccc2ncccc12', 'Oc1cccc2ncccc12', 'Oc1cccc2ncccc12', 'Clc1cc(C[NH+](Cc2ccco2)Cc2ccc(Cl)o2)co1', 'Clc1cc(/N=C/c2cnc(-c3cccs3)s2)ccc1NC[C@@H](O)c1ccccc1', 'Clc1ccccc1COCCBr', 'Cc1nonc1N(C)C(=O)[C@@H](C)Oc1ccccc1Cl', 'O=C([O-])c1nnnn1-c1cccc(OCC(F)F)n1', 'Clc1cccc([S@@]c2ccccn2)c1', 'Oc1cc(-n2nc([C@@H](OC[C@@H]3CCCO3)C2=O)c2ccccc2)ccc1F', 'O=[N+]([O-])c1ccncc1OCCN1C(=O)c2ccccc2C1=O', 'N=Cc1cn[nH]c1SCC(=O)NCc1ccccc1', 'N=Cc1cc[nH+]c(N2CCCC2)c1N[C@H]1CCO[C@]2(CO2)[C@H]1C', 'Clc1ccc(-c2nnnn2-c2ccc(F)cc2)cc1', 'CCC[C@@H](C(=O)N(C)[C@H](C)C(C)C)c1ccc(Cl)cc1', 'Clc1ccc(-c2cccs2)cc1OCCC1(F)CCCC1', 'Clc1ccc(-c2cc[nH]n2)c(NCc2ccccc2)c1', 'Oc1cccc(NCc2ccc(-c3ccccc3)cc2)c1', 'O=C(CNS(=O)(=O)/c1ccc2ccccc2c1)N1CCN(c2ccc(F)cc2)CC1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccn1Cc1ccccc1', 'C[C@@H](CNC(=O)c1cccc(Br)c1)NC(=O)c1ccccc1', 'Oc1cccc2cc(OC[C@H](O)COc3ccccc3)[nH]c(=O)c12', 'Oc1ccccc1OC[C@H](O)COc1ccc2ccc(=O)oc2c1', 'NS(=O)(=O)NCc1cc(C2CC2)ccc1[O-]', 'O=[N+]([O-])c1c(-c2nsc([C@@H]3CCOC3)n2)cnn1Cc1ccccc1', 'Clc1ccc(-c2ccccc2[N-]CCO)cc1', 'O=[N+]([O-])c1c(F)cc(N[C@@H]2CCN(S(=O)(=O)OCCCn3cccc3)C2)cc1', 'Oc1cc(F)cc([C@@]2(NC(=O)c3ccc[nH]c3=O)N3CCCC[C@H]32)c1', 'ClCc1ccn(-c2ccccc2Cl)n1', 'CCn1cnc(N[C@](C)(CO)N2CCOCC2)c1C', 'Clc1ccccc1Nc1cn(CCc2ccccc2)nc1CSc1ccccc1', 'Clc1ccccc1NC1([N+](=O)[O-])CCCCC1', 'Clc1ccccc1C(=O)OC[C@@H]1CN2CCC[C@@H]2CO1', 'Oc1cc(F)ccc1SCC(=O)N1CCCC1', 'Clc1ccc(-c2n[nH]cc2-c2ccc(NC[C@H](O)C[NH+]3CCCC3)cc2)c(F)c1', 'C=C[C@@H](NC(=O)c1ccc(F)c(C)n1)c1ccccc1Cl', 'Oc1cccc(-c2cccc(NC(=O)c3ccc(F)c(S(=O)(=O)N4CCCC4)c3)c2)c1', 'O=C(/C(c1cccs1)Cn1ncc2ccccc2c1=O)c1cccs1', 'ClC[C@H](O)COc1ccccc1', 'CN[C@H](NC(=O)c1ccccc1Cl)C(=O)NCCc1nccs1', 'Clc1cccc(NC(=S)N2CC[NH+](Cc3ccccc3)CC2)c1', 'Clc1ccc(C[NH+](CCO)[C@@H]2CCC[C@H]2S(C)(=O)=O)cc1Cl', 'OC[C@@H]([NH2+][C@@H]1CCOC2(CCCC2)C1)C(=O)NCc1ccccc1', 'Clc1ccc(Br)cc1[C@@H]([O-])CSc1ccccc1', 'O=C(/C(c1ccnc(Cl)c1)N[C@H]1CCN(c2ccccc2)C1=O)c1cccnc1', 'ClC(F)(F)c1ccccc1OCc1nccn1Cc1ccccc1', 'C[C@@H](NS(=O)(=O)c1ccccc1[N+](=O)[O-])c1ccc(F)cc1F', 'Cc1ncsc1C[C@H](CS)C[NH+]1CCCCCC1', 'Clc1cc(Br)c2nc([C@H]3[C@@H]4CCCC[C@H]4C3)c[nH]c12', 'Clc1ccsc1CN(Cc1c[nH]c2ccccc12)C1CC1', 'Clc1cc(CCl)cc(OCc2ccc(Cl)cc2)n1', 'C[C@@H](c1ccccc1)S(=O)(=O)CC(=O)NC(=O)N[C@@H](C)Cc1ccccc1', 'COc1nc2ccc(Cl)cc2cc1C(=O)NCC(=O)NC1CCCC1', 'COC[C@@H](C)NC(=O)c1ccnc(OCC(C)C)c1', 'CC[C@@H](NC(=O)Cc1cc(C(C)C)no1)C(=O)OC', 'CCc1cnc(C[NH+]2CC[C@@H](C)C[C@H]2C)c(C)n1', 'O=C(/N=c1/[nH]c2ccccc2c(=O)n1-c1ccccc1)c1csc(C2CC2)n1', 'Clc1ccc(Cc2nccc(CCCCCl)n2)cc1', 'Clc1cc(Cl)nnc1Oc1ccc(F)cc1NCc1ccccc1', 'Oc1ccccc1CC[C@H](O)C[NH2+]Cc1cc(-c2ccccc2)no1', 'Oc1ccc(F)cc1-n1ccc2cc(F)ccc21', 'Nc1[nH]nc(N2CC[NH+](Cc3cnn4ccnnc34)CC2)c1-c1ccc(Cl)cc1', 'O=C(CNCc1c(C)cc([N+](=O)[O-])c2ccc(Cl)cc12)c1ccccc1Cl', 'O=[N+]([O-])c1ccn(C2CC[NH2+]CC2)c1', 'NS(=O)(=O)Nc1cc(F)ccc1C(=O)NC[C@H](c1ccco1)[NH+]1CCCC1', 'O=C(CCOc1cccc(Cl)c1)NCc1ccccc1Cl', 'O=C([O-])[C@@H]1C[C@](O)([C@H](F)C(F)(F)F)N(Cc2ccc(Cl)cc2)C1', 'Clc1cccc(/C(c2ccccn2)N2CCN(CC(=O)Nc3ccc(Cl)cc3)CC2)c1', 'Clc1cc(F)cc2ncn(CCC[NH+]3CCCCCC3)c12', 'Clc1ccc(Br)cc1O[C@@H](c1ccccc1)C1CC1', 'Clc1ccsc1[C@H](c1ccccc1)N1CCOCC1', 'O=C([O-])c1cnc(S[C@@H]2CCS(=O)(=O)C2)n1-c1ccc(Cl)cc1', 'Clc1ccccc1CCC1CC[NH+](CCOc2ccc(C(=O)[O-])cc2F)CC1', 'Clc1ccc(C2CCC2)c(NCc2ccco2)c1O', 'Clc1ccccc1O/N=C(/[O-])C[C@H]1Cc2ccccc2S1', 'Clc1cccc2ncnc(Sc3ccccc3)c12', 'O=C(/N=C/c1ccc(-n2cccn2)cc1)N1CCC(O)CC1', 'Oc1cc(C[NH2+]Cc2ccc(F)cc2)ccc1Oc1ccccc1', 'Clc1ccsc1-c1ccc(Cl)cc1', 'Clc1cc(CNC2CC2)nc(-c2ccc(F)c(F)c2)n1', 'Clc1ccc([C@@H](Br)COc2ccccc2)cc1', 'Clc1ccc(CO)cc1OCCn1nccc1CN1CCN(Cc2ccccc2)CC1', 'Clc1nnnn1CC[NH+]1CCC[C@H]1c1c[nH]cn1', 'Clc1cccc(/C(Br)=C/CN2CCOCC2)c1', 'ClCc1nonc1NCCC[NH+]1C[C@H](C)CC[C@H]1Cc1ccccc1', 'ClCc1nc(C[NH+]2CCN([C@H](CO)c3ccccc3)CC2)cs1', 'Oc1cccc2ccccc12', 'Oc1cccc2ccccc12', 'Oc1cccc2ccccc12', 'N[C@@H]1[C@H](c2cccc(F)c2)CCN(Cc2cncc(-c3cc(OC)c(OC)cc3)n2)C1', 'O=C(c1ccsc1)c1nn(-c2ccc(F)cc2)c2ccccc12', 'Clc1cccc2c1CCC[C@H]2[NH2+]Cc1ccc(OC(F)F)cc1', 'ClC[C@H](Cc1ccsc1)Nc1ccc(Br)cc1[N+](=O)[O-]', 'Oc1ccccc1-n1cc(/N=C/c2cccs2)c2ccccc21', 'Clc1ccc(-c2nnnn2/N=C2\\S[C@H]3OCCC[C@@H]13)CC(=O)N2Cc1ccccc1', 'O=[N+]([O-])c1c(-c2cc(-c3ccc(F)cc3)n[nH]2)ncnc1C1CC1', 'Clc1nncc(N[C@H]2CCCC[C@@H]2n2cnnc2)c1Cl', 'Oc1ccccc1OC[C@@H]1Cc2ccccc2O1', 'Clc1ccccc1OCc1ccc(Cl)cc1', 'Clc1ccccc1COc1ccccc1F', 'O=C(c1c[nH]c([O-])n1)N1CCN(S(=O)(=O)c2ccc3c(c2)CCCC3)CC1', 'Clc1cccc(COc2cccc(/C(Nc3ncccn3)[NH+]3CCCCCC3)c2)c1', 'O=C(/C=C/O[C@@H](O)c1ccc(F)cc1)NCc1ccccc1', 'Clc1cc(F)cc(F)c1N1CC[NH+](C[C@H](O)CN2Cc3ccccc3C2)CC1', 'O=C(c1cc([S@](=O)Cc2ccccc2F)c[nH]1)Nc1ccc2nc(C3CC3)oc2c1', 'CCn1nc(CNC(=O)CC(C)C)c(=O)c(C)c1C', 'OC[C@@H](O)C[NH+]1CCC([C@H](O)c2ccc(Br)cc2)CC1', 'O=C(Cc1cn(-c2ccc(F)cc2F)nn1)NCc1ccccc1', 'NS(=O)(=O)Cc1noc(S(=O)(=O)N2CCN(c3ccccc3Cl)CC2)n1', 'Clc1ccccc1NC(=O)/C(=C/Cc1cccs1)c1ccccc1', 'O=C(COc1cc(Br)cnc1Cl)N(Cc1ccc(F)cc1)C1CC1', 'ClCc1csc(OC[C@@H]2C[C@H]2c2ccccc2)n1', 'O=[N+]([O-])c1c(F)cccc1NCCOCc1ccccc1', 'Clc1ccccc1[C@H]([NH3+])[C@H]1CSC(=O)N1Cc1ccccc1', 'Clc1ccc(CCNCc2cc(F)cc3c(O)cccc23)cc1', 'ClCc1ccn(COc2ccccc2)n1', 'Oc1ccc(F)cc1S[C@@H]1CC[C@@H]([NH+]2c3ccccc32)C1', 'Nc1cn[nH]c1[C@@H]1N=C(C2CC2)c2nc3ccc(Cl)cc3n21', 'Clc1ccccc1[C@H]([NH3+])c1cc(-c2ccc(Cl)cc2)nn1CCO', 'Clc1ccc(C2CC2)cc1', 'Clc1ccc(-c2n[nH]cc2CSc2n[nH]c(=O)n2CCc2ccccc2)cc1', 'Oc1ccc(/C=N/Sc2nnc(N/N=C\\c3ccc(Br)cc3)o2)cc1', 'O=C(/N=C/C=C\\c1cccc2c1OCO2)c1ccccc1', 'C[C@@H]([NH2+]CC(=O)NC1(Cc2cccs2)CCC1)c1cccc(Cl)c1', 'ClCc1ccc2c(c1)OCC[C@@H]2c1ccc(CN2CCCCC2=O)cc1', 'C=C[C@@H](O)[C@H](C)c1cccs1', 'Clc1ccccc1OCc1csc(C(=O)N[C@@H](CCS2CCOC2)C(F)(F)F)c1', 'Clc1ccc(Br)cc1/C=N\\c1ccc(C(C)C)cc1', 'Oc1ccc(/C=N\\N(Cc2ccccc2)S(=O)(=O)c2ccccc2)cc1F', 'Oc1ccccc1OSCC(=O)N1CCOC[C@@H]1Cc1ccccc1', 'Clc1ccccc1Cc1nc(Cc2[nH+]cc(Cl)cc2Cl)n[nH]1', 'Clc1ccc(COCO[C@H]2CCC[NH2+]C2)cc1', 'Clc1cc(COC[C@H]2CCCN(c3ccccc3F)C2)sc1NC1CC1', 'Nc1n[nH]c2ccccc12', 'Clc1ccc(-c2cc(SCc3nc4ccc(F)cc4o3)n[nH]2)cc1', 'O[C@@H](c1cc(Cl)cc(Cl)c1Cl)c1ccccc1C1CC1', 'Clc1ccc(-c2nonc2NC2CCCC2)o1', 'NS(=O)(=O)N[C@@H](c1ccccc1Cl)C1([NH+]2CCOCC2)CCCC1', 'Clc1ccccc1NS(=O)(=O)N1CCC(CN2CCCCCC2)CC1', 'CNS(=O)(=O)C[C@H]1CCCN1C(=O)CCc1ccccc1OC', 'Oc1cc(F)ncc1CN(Cc1ccco1)CC(F)(F)F', 'O=[N+]([O-])c1ccnc2ccc(OC(=O)C3(c4ccccc4)CCC3)cc12', 'O=C(Cc1n[nH]cc1C(=O)[O-])Nc1cc(C(F)(F)F)cc([N+](=O)[O-])n1', 'Oc1cccc(CC2(O)C[C@@H]2c2ccco2)c1O', 'Clc1ccc(Cc2noc(C(=O)N3CCCCC3)n2)cc1', 'O=C([C@H]1c2ccccc2S(=O)(=O)C[C@@H]1NC(=O)Cc1c[nH]c2ccccc12)N1CCCCC1', 'CC[C@@H](COC)S(=O)(=O)c1ccc(OC)c(OC)c1', 'O=C(/C=C/c1c(O)oc2cccnc12)COC1CCCCC1', 'O=[N+]([O-])c1cc(N[C@@H]2CCN(c3ccc(F)cc3F)N2)cs1', 'Clc1ccc(Br)c(SCc2ccccc2)c1', 'O=C(COc1ccc(F)cc1)NNC(=O)c1ccccc1Br', 'Clc1ccccc1-c1csc(CN2CCN(Cc3ccccc3)CC2)n1', 'NS(=O)(=O)NCCOC1(c2ccccc2)CCC1', 'NS(=O)(=O)CC[C@@H](NC1CC1)c1ccccc1F', 'Clc1ccccc1COCc1cccc(COc2ccccc2)c1', 'Oc1ccccc1[C@@H]1OCC[NH+](CN2C(=O)c3ccccc3C2=O)C1', 'OCc1n[nH]c(-c2ccc(N3CCCC3)c[nH+]2)c1[N+](=O)[O-]', 'O=C(/C=C/c1ccc2c(c1)OCO2)c1cnn(Cc2ccccc2)c1', 'N[C@@H](S/C=C/c1ccc(Cl)cc1)c1nccn1C', 'O=[N+]([O-])c1ccc(OCC(=O)N2CCCC[C@H]2c2ccccc2)cc1', 'O=C(/N=C/N=C/c1cccc(Cl)c1)NCCOc1ccc(F)cc1', 'N[C@@H](CNC(=O)c1cccn2ccnc12)c1cccnc1', 'Clc1ccccc1-c1noc(Cc2ccccc2)n1', 'Clc1cc(/C=Cc2n[nH]c(-c3ccc([N+](=O)[O-])cc3)n2)ccc1N1CCOCC1', 'O=[N+]([O-])c1c(Br)cc(F)cc1NC[C@H](O)c1ccc(Cl)cc1', 'Clc1cc(NC[C@@](O)(c2cccc(F)c2)n2nc(C(F)(F)F)cc2)ccc1Cl', 'Clc1cc(Br)c(C[NH3+])nc1NC[C@@H]1CCCO1', 'Clc1ccc(-c2n[nH]c3c2C(=O)C[C@@H](c2ccccc2)C3)cc1', 'O[C@@H](c1cc(I)ccc1F)[C@@H]1CN(C(F)(F)F)CCO1', 'Oc1ccccc1COc1ccccc1Oc1ccc(F)cc1F', 'O=[N+]([O-])c1c([O-])c(COc2cccnc2)c2ccccc2[n+]1[O-]', 'O=C([O-])/C(=N/Nc1ccccc1Cl)c1ccncc1', 'CCC(CC)C(=O)N[C@H](C)c1ccc(OC)cc1', 'Clc1ccc(Br)c(F)c1CN[C@@H](c1nccs1)c1ccccn1', 'NS(=O)(=O)Cc1noc(Cn2c(=O)ccc3nc(C(=O)[O-])cn32)n1', 'Clc1ccc(-c2csc3[nH]c(Nc4cccnc4F)nc23)cc1', 'Clc1ccc(-c2nnnn2[C@@H]2C[C@@H]2CO)cc1', 'Clc1cc(Br)cn1[C@@H]1CCC[NH+](C2CCCCC2)C1', 'COc1ncncc1C[C@@H]([NH3+])Cc1cc(Cl)ccc1Cl', 'Oc1cc(CCNC2(CS)CCOCC2)ccn1', 'O=[N+]([O-])c1ccncc1[C@@H]1c2ccccc2C(=O)N1CN1C(=O)c2ccccc2C1=O', 'NS(=O)(=O)Cc1nccc(CO)c1C(C)C', 'O=C(/C(c1cccs1)c1ccccc1Cl)[C@H]1C[C@H]1c1cccc(Cl)c1', 'Clc1cccc([S@@]c2c[nH]c3ccccc32)c1', 'O=C(/C(c1cccs1)CN1CCOCC1)C(=O)Nc1cccc(Br)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(CN(Cc2ccccn2)c2ccc([N+](=O)[O-])cc2)c1NCC1CC1', 'ClC(c1cccs1)c1cc(N2CCN(C(=O)Cc3ccccc3)O2)ccn1', 'Clc1cc(/N=C/c2ccc(I)cc2)nc(Oc2ccc(O)cc2)c1', 'Clc1cc(/N=C/c2nc3ccccc3o2)ccc1Oc1ccc([N+](=O)[O-])cc1[N+](=O)[O-]', 'Clc1cccc2cc(CNS(=O)(=O)c3ccccc3Cl)[nH]c12', 'Clc1ccccc1Cc1n[nH]c([C@H]2CCc3ccccc32)n1', 'Clc1ccccc1Oc1cnc(CBr)cc1', 'Oc1ccc(F)cc1-n1cccc1CO/C=C/c1ccccc1Cl', 'O=C(CNCc1nnnn1-c1cc(F)cc(Cl)c1)NCc1ccc(F)cc1', 'Cc1sc2ccccc2c1C(=O)N(C)[C@@H]1CCC[C@]2(CCOC2)C1', 'CNS(=O)(=O)[C@@H]1CCN(C(=O)[C@H]2CCCN2C(=O)Cc2ccccc2)C1', 'Nc1[nH]nc2nc(COc3ccc(F)c(Cl)c3)[nH]c12', 'Clc1cc(/C=N/N2CCOCC2)ccc1OC(=O)Cc1ccccc1', 'Clc1cccc2nn(S[C@@H](c3ccccc3)c3ccc4ccsc4c3)nc12', 'O=C(CS(=O)(=O)c1ccccc1)NCC1(C2CC2)CCCCC1', 'Clc1cccc2ccc(N3CCO[C@@H](c4cc(Cl)cc(Cl)c4)C3)nc12', 'CNS(=O)(=O)CCNC(=O)NCc1cc(F)c(C)s1', 'Clc1cccc2[nH]nc(N/N=C/c3ccccc3)c12', 'Clc1cccc(CNC[C@H]2CCC[C@@H]2n2cccn2)c1', 'Clc1ccc([C@@H](NSc2nncc3c2CCC3)C(F)(F)F)cc1', 'Oc1cccc(-c2cc(C3CC3)nc3ccccc23)c1', 'ClCc1cn[nH]c1C1CCN(c2cccnc2)CC1', 'Clc1nncc(Cl)c1N[C@H]1COCCN1CC1CC1', 'O=C(c1cccs1)[C@H]1Cc2ccccc2CN1C(=O)c1ccco1', 'Nc1[nH]nc(C2CC2)c1Br', 'ClCc1n[nH]c2c1CCC2', 'ClCc1n[nH]c(C2CC2)n1', 'Clc1ccc(Br)c(-n2nc(COC(=O)C3CCCC3)cc2Cl)c1', 'Oc1cc(O[C@@H](C)C(=O)N2CCC[C@H]2C[C@@H]2CCCO2)ccc1Cl', 'Clc1ccccc1Oc1cnn(C[C@@H]2CCCC[C@@H]2O)c1', 'Clc1cc(Cl)nnc1-n1cnc2ccccc21', 'Clc1ccc([C@@H](OC(=O)C2Oc3ccccc3F)C2)cc1', 'Clc1ccc(CCC[NH2+]Cc2ccc[nH]2)cc1Cl', 'Clc1ccccc1CNc1nc(COc2ccccn2)cs1', 'Clc1ccc([C@@H](O)c2ccccc2)nc1', 'Clc1ccc(C(=O)N2CCN(Cc3c[nH]nc3C3CC3)CC2)c(F)c1', 'Clc1ccccc1OC1CN(C(=O)c2cc(OCc3ccco3)on2)C1', 'Clc1ccccc1CO[C@H](O)c1cscc1[N+](=O)[O-]', 'ClCc1cn(Cc2ccccc2)nc1-c1cccnc1', 'Cc1nonc1CCCC(=O)NNC(=O)C[C@@H](O)c1ccccc1C', 'Clc1cc(Nc2nc3cc(Cl)ccn3n2)c2c(nc3ccccc13)CCC2', 'Clc1cc(Cl)c(CN(Cc2ccccc2)C(F)(F)F)cc1O', 'O[C@@H]([C@@H](CCc1ccsc1)NCCCc1ccc2c(c1)OCCO2)C1CC1', 'COc1ncc(/C=C/C(=O)Nc2ccc(Cl)nn2)c(C)c1CC', 'Clc1ccccc1-c1n[nH]c2cc[nH]c12', 'Clc1ccc(-c2cc[nH]n2)o1\n\n\n\n\n', 'Clc1ccc(-c2cc[nH]n2)o1\n\n\n\n', 'Clc1ccc(-c2cc[nH]n2)o1\n\n\n', 'Clc1ccc(-c2cc[nH]n2)o1\n\n', 'Clc1ccc(-c2cc[nH]n2)o1\n', 'Clc1ccc(-c2cc[nH]n2)o1', 'Clc1ccc(-c2cc[nH]n2)o1', 'Clc1ccc(-c2cc[nH]n2)o1', 'Clc1ccc(-c2cc[nH]n2)nc1', 'Clc1ccc(-c2cc[nH]n2)nc1', 'Clc1ccc(-c2cc[nH]n2)cn1', 'Clc1cccc([C@@H](NCc2cc3nc[nH]c3c(Cl)c2Cl)C2CC2)c1', 'OC[C@H](Nc1cccc(F)c1)C(=O)N1CCCC1', 'Clc1ccc([C@@H](NNc2nc3ccccc3s2)C2CC2)cc1', 'Clc1ccc(-c2cc([C@H](NC3CC3)C(=O)N3CCCC3)ccc2Cl)cc1', 'Clc1cccc([S@@](=O)CC(F)F)c1O', 'Clc1nnc(-c2ccccc2)n1COc1ccccc1', 'Cc1nonc1C(=O)N(CC1CCOCC1)[C@H]1CCS(=O)(=O)C1', 'Clc1ccc(Br)cc1/C=N\\N=C\\C=C\\c1ccc(Br)cc1', 'ClCc1cn(Cc2ccc(F)cc2)nc1S[C@@H]1CCSC1', 'CCn1nc2n(c1=O)CCC[C@@H]2[NH2+][C@H]1c2ccccc2C[C@H]1OC', 'Oc1ccc(/C=N\\CCCCSc2ccccc2)cc1', 'O=C(COc1ccc2c(c1)CCC(=O)N2)N1CCN(c2ccccc2)CC1', 'Clc1nncnc1[N-][C@H]1CC[C@@H]2CCCC[C@@H]21', 'Clc1ccccc1O/N=C\\C(=C\\Br)c1ccc(Br)cc1', 'NS(=O)(=O)N(Cc1ccc(Cl)cc1)C1CCSCC1', 'Clc1ccsc1NCc1ccc(F)c(S(=O)(=O)N2CCCCC2)c1', 'O=C(CNc1nnc(Cl)c2ccccc12)Nc1ccccc1Cl', 'O=[N+]([O-])c1c(Br)cnc(Oc2ccccc2Cl)c1C(F)c1ccccc1', 'Clc1ccc(-c2nonc2C(F)F)cn1', 'Oc1ccc([N+](=O)[O-])c(/N=N\\Nc2nnc(SCC(F)(F)F)s2)c1', 'Clc1cccc2[nH]nc(S/C=C/C3CCCCC3)c12', 'O=C(c1c[nH]c(=O)[nH]c1=O)N[C@@H]1CCS(=O)(=O)C1', 'O=C([O-])/C(Cc1ccccc1)=N\\NC(=O)c1ccc(C(F)(F)F)cc1', 'Clc1ccc([C@@H](CNCc2ccccc2)N2CCCCC2)c(Br)c1', 'Clc1ccccc1OCc1cc(Br)ccc1Br', 'COC[C@H](NCc1ccccc1C)[C@H]1CCOC1', 'O=C(/C(c1cccs1)C(=O)c1cccs1)c1ccc(OC(F)(F)F)cc1', 'Clc1ccc(Br)c([C@H]2CCC[NH+]2Cc2nnco2)c1', 'Clc1cc(Br)ccc1C[NH+]1CCC[C@@H](CCO)C1', 'O=C(CCOn1ccccc1=O)Nc1cc2c(cc1Cl)OCCO2', 'O=[N+]([O-])c1ccc[nH+]c1I', 'Oc1cc(C=O)ccc1OCCN1CCOC[C@@H]1C1CC1', 'O=[N+]([O-])c1c(Cl)nc(-c2ccc(OS(=O)(=O)C3CCCC3)cc2)c2ccccc12', 'Clc1ccc([C@@H]([O-])C[C@H](O)Cc2cc(Cl)ccc2F)cc1', 'Clc1ccc(CC[NH+]2CCC(c3nccs3)CC2)cc1', 'Clc1ccccc1-c1csc(C[NH+]2CCCCC2)n1', 'Clc1cccc2ncnc(N3CCOC4(CCCC4)C3)c12', 'O=C(c1c[nH]nc1-c1ccc(Br)cc1)N1CCN(S(=O)(=O)c2ccc(F)c(F)c2)CC1', 'Clc1cc(CO[C@H]2CSCCS2)c(O)cc1O', 'Clc1ccc(Br)cc1/C=N\\O[C@@H]1CCCO[C@@H]1c1ccccc1', 'Oc1ccc(F)cc1Oc1ccc(NCc2ccc(Cl)nc2)cc1', 'Clc1cc(Cl)ncc1O[C@@H](C(=O)c1ccccc1)c1ccccn1', 'Oc1cc(Cn2cccn2)ccc1OCc1ccc(F)cc1', 'Clc1ccsc1NC(=O)Cc1c[nH]nc1Cc1ccccc1', 'Clc1ccccc1Cc1c(C(=O)[O-])ncnc1NO', 'Oc1cc(Cn2ncccc2=O)ccc1COC1CCN(C(=O)c2ccccc2)CC1', 'NS(=O)(=O)N[C@H]1c2ccccc2C(=O)N1CC(N)=O', 'ClCc1ncnc(N2CCOCC2)c1-c1ccc(F)cc1', 'O[C@@H]1C(/C=N/O)C=CC(=O)C1(CO)CCc1ccccc1', 'OCS/C(=C/C=C/O)C1(c2ccc([N+](=O)[O-])cc2)CC1', 'Oc1cc(OC[C@@H](O)C[NH2+]C2CC2)ccc1OCc1cccnc1Cl', 'O=C(C1CCN(S(=O)(=O)c2ccc(F)cc2)CC1)N1CCN(C(=O)c2cccc(F)c2)CC1', 'Cc1nonc1NC(=O)/C=C/c1cc(C)cc([N+](=O)[O-])c1', 'CCn1nnc(CNC(=O)c2ccc(F)cc2Cl)n1', 'Clc1cc(N2CCO[C@@H](c3ccccc3F)C2)nc2ccccc12', 'O=C(Cc1noc2ccccc12)Nc1cccc(Sc2nc3ccccc3s2)c1', 'Clc1ccc(Br)cc1S[C@H]1c2ccccc2C1(Cl)Cl', 'O=C([O-])/C=C/C(=O)N(Cc1ccccc1)c1ccccc1', 'O=C(/C=C/OCc1ccccc1O)c1ccccc1F', 'Clc1cc(Br)cc(/C=N/NCC2[NH+]c3ccccc3C2=O)c1', 'Clc1ccc(-c2nnnn2Cl)cc1', 'Clc1ccc(-c2nnnn2Cl)cc1', 'Clc1ccc(-c2nnnn2Cl)cc1', 'Clc1ccc(-c2nnnn2Cl)cc1', 'Clc1ccc(-c2nnnn2Cl)cc1', 'O=C(C1=CCN(c2ccc(C#N)c(Cl)c2)CC1)N(Cc1ccccc1)Cc1ccccc1', 'C[C@@H](NC(=S)Nc1nccs1)C(=O)NC1CC1', 'O=C(CNc1nn(Cc2ccncc2)nc1Cl)Nc1cccc2c1C[C@@H](O)C2', 'Clc1cccc(CCOCCCOc2ccccc2F)n1', 'Clc1ccccc1-c1csc(/C=C/c2ccncc2)n1', 'O=[N+]([O-])c1ccc[nH+]c1[C@@H](N1CCOCC1)[C@@H]1CCS(=O)(=O)C1', 'ClCc1cc([N+]([O-])=Nc2ccccc2Cl)oc1N1CCCC1', 'Oc1ccc(O[C@H](C(=O)NC2CCCCCC2)C(F)(F)F)cc1', 'Clc1nnnn1/N=C/[C@@H]1CCCC[C@H]1O', 'Oc1cc(C(F)(F)CN2CCOCC2)cc(Cl)c1OCc1ccccc1', 'Clc1nnnc(NC[C@@H]2CCCO2)n1', 'Clc1nnn(C[C@@H](O)CO)c1Br', 'Clc1ccc(Br)c([C@H]([NH2+]Cc2cccc(Cl)c2)[C@H]2CCSC2)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1C(=O)N[C@H]1CCSc2ccc(F)cc21', 'Clc1cc(NCc2ccc(C[NH+]3CCCC3)cc2)ncn1', 'Clc1ccccc1C1(Oc2ccccc2F)CC1', 'O=C(COc1ccccc1C(=O)[O-])NCc1ccccc1', 'OCS[C@@H](c1ccncc1)c1ccc(Cl)cc1', 'Clc1ccc(-c2n[nH]c(OCC(F)F)n2)cc1', 'Clc1ccc(Br)cc1OC/C=C/c1ccccc1', 'Clc1ccccc1OC1CCCC1', 'O=C(c1cc(Br)c[nH]1)N(Cc1ccncc1)Cc1cccnc1', 'CCc1cccc(Oc2ccccc2NC(=O)Cc2ccccc2)c1', 'O=C(COc1ccc2nnnn2c1)NCc1cccc(C[NH+]2CCCC2)c1', 'Oc1ccc([C@H]2NC(=O)N/C(=C\\c3cccc(O)c3)N2Cc2ccccc2)cc1', 'Oc1ccc(F)cc1-n1cccc1C(=O)NCc1ccccc1Br', 'O=C([C@H](SCc1ccccc1O)Br)c1ccccc1', 'Clc1nnc(-c2ccccc2)nc1CS[C@@H]1CCCc2ccccc21', 'O=C(CNS(=O)(=O)C1CCCCC1)NNc1cc(C#N)ccc1O', 'CCCC[NH+](C)CC1CCN(c2nccnc2C#N)CC1', 'ClCc1ccn(CCn2cccn2)c1', 'Clc1ccc(-c2nnnn2CC2CC2)cc1', 'Clc1ccc(COc2ncc(Cl)cc2CO)cc1', 'O=C(c1c[nH]nc1C[NH+]1CCC[C@@H]1c1nccs1)N1CCCC1', 'ClCc1nonc1COc1ccccc1', 'CN[C@H](Cn1cncn1)c1ccncc1', 'O=[N+]([O-])c1ccn([C@H]2CCCN2S(=O)(=O)c2ccccc2Br)c1', 'O=C(c1cncc(Br)c1)N1CCN(c2[nH+]ccn2Cc2ccccc2)CC1', 'ClCc1cc(Br)cnc1OCCn1cccnc1=O', 'O=C(c1c[nH]c2cc(S(=O)(=O)N3CCOCC3)ccc12)N1CCC[C@H](O)C1', 'Clc1ccc([C@@H]([O-])N(C[C@H]2CCOC2)C2CC2)cc1', 'Clc1ccc(C(=O)COC[C@@H]2CCOC2)cc1', 'NS(=O)(=O)N(C[C@H]1CCCN1S(=O)(=O)c1ccccc1)c1ccccc1', 'Clc1nnnn1c1ccc(OCC(N)=O)cc1', 'CC1CCC([C@@H](C)NC(=O)Nc2ccc(Cl)cc2)CC1', 'Clc1nnn(-c2nc(C3CC3)ns2)c1OCc1ccccc1', 'O=[N+]([O-])c1ccc(-c2n[nH]cc2CC(=O)N2CCCC2)cc1', 'Clc1cccc(S[C@H](C[C@@H]2CCCCC[NH2+]2)c2cccs2)c1', 'O=C([O-])CC[C@@H]1COc2ccccc21', 'Clc1ccc(-c2cc[nH]n2)c(/C=C/c2ccccc2)c1', 'OC1=C(Nc2nc([O-])c(-c3ccccc3)n3c4ccccc4n23)C2(CCC[NH2+]CC2)C1', 'Clc1ccccc1C[NH+](CCCN1CCOCC1)C1CC1', 'O=C(/C=C/c1ccco1)N1CCC[C@H](C2(O)COC2)C1', 'O=C(CS(=O)(=O)[C@@H](c1ccccc1)N1CCOCC1)Nc1ccc2c(c1)OCCO2', 'NS(=O)(=O)/C=C(\\c1ccccc1)N1CCN(Cc2ccc3c(c2)OCCO3)C1', 'Cc1cnc([C@@H]2[C@@H](c3ccccc3)OC[C@@H]2NC(=O)/C=C/c2ccccc2)s1', 'CCCn1nnnc1COc1cccc(Br)c1C', 'N#CCc1ccc(CNC(=O)Nc2nc(C3CC3)n(-c3ccccc3)n2)cc1', 'O=[N+]([O-])c1ccc(S[C@@H](C(=O)N2CCCC2)c2nccs2)cc1', 'Clc1cc(/C=C/C(=O)N2CCN(Cc3ccccc3)CC2)ccc1O', 'Clc1ccc(-c2cc[nH]n2)nc1[C@@H]1CCCN(C(=O)Nc2ccccc2)C1', 'Clc1cc(Cl)nn1CCOC1CC1', 'Clc1nnn(Cc2ccc(F)cc2Cl)c1NOC(=O)CC1CCCC1', 'O=C(CCOC1CCCCCC1)c1ccc(F)cc1', 'C=C[C@@H]([NH3+])c1ccc(OC[C@@H]2CCCO2)cc1', 'O=C(COc1cc([N+](=O)[O-])ccc1OCc1ccc([N+](=O)[O-])cc1)Nc1ccc([N+](=O)[O-])cc1', 'Clc1ccccc1NC(=O)CSc1nnc(-c2ccccc2)n1C[C@@H]1CCCO1', 'C[C@@H](C[NH+]1CCCCC1)N[C@H](C)c1ccc(CO)o1', 'Nc1[nH]nc2c1[C@@H]1CCC[C@H]1C2', 'OC[C@@H](c1ccccc1)N1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1', 'O=C([C@H](NS(=O)(=O)c1ccc(Cl)cc1)c1ccccc1)N1CCC[C@H](CO)C1', 'N#C/C(=C\\c1cccc(NC(=O)c2cnn(C3CCC3)c2)c1)C(N)=O', 'Clc1ccccc1Oc1cc(Oc2cc(C(F)F)ncn2)ccc1O', 'Clc1ccccc1O/N=C/c1cc(Br)ccc1OCn1cccn1', 'Clc1ccc(Br)cc1O[C@@H](Nc1ccc(F)cc1)n1cccn1', 'Clc1ccc(-c2nonc2)nc1Nc1cc(Br)ccc1OC(F)F', 'Oc1ccccc1Oc1ccc2c(=O)nc(NCCO)oc2c1', 'Clc1cc(Cl)cnc1N1CCCCC[C@@H]1C[C@@H]1CCCO1', 'C=CCC[C@@]1(Cc2cccc(OC)c2)CCCN(c2ccc(N(C)C)cc2)C1', 'Clc1ccccc1OCCS(=O)(=O)C1CC1', 'Clc1ccccc1[C@H](c1nc2ccccc2s1)N1CCc2ccccc2C1', 'Cc1nonc1C[NH+]1CCC(NC(=O)c2ccc(C[NH3+])cc2)CC1', 'Oc1cc(C[NH2+][C@H](c2ccccc2)c2ccc(F)cc2)c[nH]1', 'O=C(c1c[nH]nc1N1CCOCC1)N1CCN(Cc2cccc(C(F)(F)F)c2)CC1', 'Clc1ccc(Cc2csc(-c3c[nH]nc3N3CCCC3)n2)cc1', 'Clc1ccc(-c2cccs2)nc1OC1CCN(C(=O)c2ccccc2)CC1', 'Nc1[nH]ncc1CN(c1ccccc1)S(=O)(=O)/N=C/c1ccc(Br)cc1', 'O=C(C1CCN(C(=O)C2CCCC2)CC1)N1CCc2ccccc2C1', 'CNS(=O)(=O)[C@@H](C)C(=O)NCc1ccccc1', 'O=[N+]([O-])c1cc(Cl)c(N2CCc3ccccc3N2)cc1[N+](=O)[O-]', 'Clc1nnnn1c1ccc(CSc2nnc3ccccn23)cc1', 'Clc1cccc2[nH]nc(NC[C@@](CO)[N+](=O)[O-])c12', 'OC[C@@H](Nc1ncccc1C(F)(F)F)N1CC[C@@H](C)[C@@H](O)C1', 'Clc1cccc(-c2ncco2)c1N[C@H]1CCCc2ccccc21', 'Clc1ccsc1/N=C(\\[O-])COc1ccc(C(F)(F)F)cc1', 'Oc1cc(OCCSc2ncc(-c3ccccc3)n2Cc2ccccc2)ccc1Oc1ccccc1', 'Clc1ccccc1Oc1cncc(-c2ccccc2)[nH+]1', 'O=C(c1c[nH]c2cc(Cl)ccc21)N1CC=C(c2ccccc2)CC1', 'Clc1ccc(-c2ccccc2[N-]S(=O)(=O)c2cc(Br)ccc2F)cc1', 'O=[N+]([O-])c1ccc2occc2c1-c1ccccc1', 'ClCc1ccnn1Cc1c[nH]c(=O)nc1N1CCCC1', 'Nc1nnc(Cl)n1C1CCCCC1', 'Clc1nnn(CC2CCCCC2)n1', 'Clc1nncn1[C@@H]1CCCC[C@H]1C(=O)[O-]', 'Clc1nnc(C2CCCC2)n1N1CCCC1', 'Clc1ccc([C@@H]([O-])N2C[C@@H](CNc3ncccc3C#N)CC2)cc1', 'Clc1ccc([C@@H](Br)[C@@]2(O)CCSC2)cc1', 'O=[N+]([O-])c1c(Br)scc1N1CCn2c1nn2-c1ccc(Br)cc1', 'Clc1ccc(Br)cc1/C=N/NC(=O)C(c1ccccc1)c1ccccc1', 'O=C(C1CC1)C1=CC[NH+]([C@H]2CCN(c3cc(F)ccc3F)C2)CC1', 'Oc1ccc([N+](=O)C=C(Cl)Cl)cc1SCCO', 'Clc1ccccc1-c1noc(CN2CCCOCC2)n1', 'Oc1cc(Br)cnc1S[C@@H]1C[C@@H](Br)CO1', 'Clc1ccccc1OCc1cccc(COCCO)n1', 'Clc1ccc(-c2csc3[nH]c(NC[C@@H]4CCCO4)cc23)cc1', 'Oc1cc(F)ncc1CNC(=O)CSc1ncnc2c1CCCC2', 'Clc1ccccc1Cc1noc(Cc2ccc(Br)s2)n1', 'Clc1cccc(/N=N\\Cc2ccc[nH]2)c1F', 'Clc1cccc(F)c1O[C@@H]1CCO[C@]2(CCSC2)C1', 'Oc1ccccc1-c1noc([C@H]2CCOC3(CCCC3)C2)n1', 'Clc1cc(N2CCC(C[NH+]3CCCC3)CC2)nc([O-])c1', 'ClCc1ncnc(C2CC2)n1', 'Clc1cc(Cl)nnc1O[C@H]1CC[C@@H]([NH2+]CC(F)(F)F)C1', 'O=[N+]([O-])c1cc(Nc2ccc(-n3cccn3)cc2)cc(S(=O)(=O)N2CCCC2)c1', 'O=C(/C=C/O)O[C@@H]1CCc2cc(OC(=O)[C@@H]3CCCCC[C@@H]3S(C)(=O)=O)ccc21', 'ClC[C@H](CS(=O)(=O)c1ccc(Br)c(F)c1)NC(=O)c1ccc2c(c1)OCCO2', 'O=C([O-])/C(=C/c1ccccc1)c1ccc(O)cc1', 'Clc1ccccc1Oc1cnc(Oc2ccc(Br)cc2F)cn1', 'Clc1cc(Cl)c2sc(-c3ccc(CCN4CCOCC4)o3)nc2c1', 'Clc1cc(/C=Cc2ccccn2)ccn1', 'O=C(c1cc(Cl)ccc1Cl)N(Cc1ccccc1)Cc1ccccn1', 'C[C@@H]([C@@H](C)CNC(=O)C1CCC1)NC(=O)c1cccs1', 'O=C(/C=C/c1c(NN)cc(C#N)c(=O)c2ccccc12)Nc1ccc(Oc2ccccn2)cc1', 'Clc1ccc(Cc2ccco2)nc1', 'Clc1ccccc1[C@H]1[C@H](CO)Cc2ccccc2C1', 'Clc1nncc(F)c1C(F)(F)CC(F)(F)F', 'ClCc1csc(N[C@H](CO)CO)n1', 'Clc1nnc(-c2ccccc2)o1', 'Clc1ccccc1Oc1cnc2ccccc2n1', 'Clc1ccc(-c2nonc2)cc1', 'Clc1ccc(-c2nonc2)cc1', 'Clc1ccc(-c2nonc2)cc1', 'C=C[C@@H](NS(=O)(=O)[C@H]1CCCN(c2ncc[nH]2)C1)c1ccccc1', 'Clc1cc(NSc2ncc(-c3ccccc3Cl)n2C2CC2)cc2c1CCCC2', 'ClCc1cccc(-c2nc3ccccc3n2Cc2ccccc2)c1', 'CN(c1ccn([C@H]2CCCCN2C(=O)C(=O)Nc2cccc(Br)c2)c1)C(C)C', 'Nc1[nH]ncc1S(=O)(=O)N1C[C@H]2CC=CC[C@@H]2C1', 'Clc1ccc(Br)cc1S[C@H](C1=c2ccccc2=[NH+]C2)C=C[C@@H]12', 'CNS(=O)(=O)[C@@H]1CCN(C(=O)Nc2ccc(-c3ncccn3)cc2)C1', 'O=C([O-])/C=C(/N=C\\c1[nH]nc(C(F)(F)F)c1Cl)NCc1ccccc1', 'Clc1nnnc(CN2CCC[C@@H]2c2ccccc2)n1', 'Clc1ccc(COCOc2ccc3ccccc3c2)cc1Cl', 'Clc1ccc([C@@H]([O-])[C@H]([NH2+]CC(=O)NC2CC2)C2CC2)cc1', 'Oc1cc(O)c(-c2ccccc2)nc1OC[C@H]1CCCO1', 'O=C(CN(Cc1ccccc1)c1ccccc1)Nc1nncs1', 'NS(=O)(=O)NC[C@H](c1ccc(Cl)cc1)N1C(=O)COc2ccccc21', 'O=[N+]([O-])c1c(-c2cccnc2)cnnc1O[C@@H](CN1CCOCC1)c1cccs1', 'Clc1ccccc1CCCOc1cccc(Br)c1', 'Clc1ccc(-c2nnnn2-c2ccccc2)c2c1CCC2', 'O[C@@H](Sc1n[nH]c2c1CCCC2)c1cccc(OC2CC2)c1', 'O=C(c1cc(Br)c[nH]1)c1cc(F)ccc1OCCN1CCCC1=O', 'NS(=O)(=O)CCCSc1nc2cc(Cl)ccc2c(=O)[nH]1', 'Oc1cc(CC[C@@H]2CCC[NH2+]C2)ncn1', 'O=C(c1cnc(-c2ccccc2)o1)N(Cc1ccccc1F)[C@H]1CCS(=O)(=O)C1', 'Clc1ccccc1OCC[NH+]1CCN(c2nc3ccccc3s2)CC1', 'Clc1ccc(-c2cc[nH]n2)c(OCCC(=O)[O-])c1', 'Clc1cc(COCOc2ccc(Cl)cc2)ncn1', 'O=C(c1cn2ccsc2n1)N1CC(C2=Cc3ccc(F)cc3C2)C1', 'ClCc1csc2nc([C@@H]3[C@@H](Cc4ccccc4)C(=O)N3)sc12', 'Clc1cccc(S(=O)(=O)/N=C(\\[O-])[C@@H]2CCOC3(CCCC3)C2)c1', 'Clc1ccccc1Nc1ccn(-c2ccc(Br)cc2)n1', 'CN(c1cccnc1)[C@H](C(=O)c1ccc(Cl)c(F)c1)C[C@H]1CCCO1', 'O=C(c1cn(-c2ccccc2)nn1)O[C@@H](c1cccs1)C1CC1', 'Oc1ccc(/C=C/Nc2ccc(S(=O)(=O)N3CCOCC3)cc2)cc1', 'O=[N+]([O-])c1c(O[C@@H]2CCCC[C@@H]2C(=O)[O-])ccc2cccnc12', 'Clc1nncc(O[C@H]2CCN(c3ccccc3Cl)C2)n1', 'Clc1ccc(Br)c(N2CSCC2)c1', 'Oc1ccccc1/C=N/N(Cc1c(F)cccc1Cl)[C@@H]1CCS(=O)(=O)C1', 'O=C(CC(CC1(Cl)CC1)NCCCS(C)(=O)=O)C(=O)Nc1ccccc1', 'NS(=O)(=O)/C(CNC(=O)CCc1ccccc1)[C@H]1CCCC[C@@H]1C(=O)[O-]', 'Clc1cccc(Nc2cc(N[C@H]3CCOC3)nc(-c3ccccc3)n2)c1', 'O[C@@H](Sc1nncn1Cc1c[nH]nc1c1ccccc1)C1CCOCC1', 'NS(=O)(=O)Cc1nnc(-c2ccc(N3CCCC3)cc2)n1N', 'O=C(C1CCN(C(=O)c2ccccn2)CC1)N1CCCC1', 'Nc1cn[nH]c1[C@@H]1CCCN1C(=O)C1CCCC1', 'Clc1nnn(-c2ncn[nH]2)c1CNC(=O)c1ccc[nH]1', 'Clc1nncc(CO/N=C/c2ccccc2Cl)n1', 'Clc1ccccc1O/Cn1cc([N+](=O)[O-])cn1', 'Clc1ccccc1OCC[NH2+][C@@H]1CCC[C@H](S(C)(=O)=O)C1', 'CCS(=O)(=O)CCN(C)C(=O)CCc1ccccc1Cl', 'Clc1cc(Br)ccc1Sc1ccc(C=O)cc1', 'NS(=O)(=O)Nc1cc(NC(=O)c2ccc3c(c2)CCC3)ccc1Cl', 'NS(=O)(=O)CCCc1cccc(C(=O)N(C2CC2)C2CCCCC2)c1', 'Nc1[nH]ncc1C(=O)Nc1cc2c(cc1Cl)c(=O)[nH]n2C', 'Clc1ccc(-c2cccs2)c2c1C[C@@H](Cl)[C@H]2Br', 'Clc1ccc(-c2nonc2N2CCCCC2)cc1', 'Clc1ccc(-c2n[nH]cc2Cn2cnc3ccc(F)cc32)cc1', 'Clc1ccc(C(CO)Oc2cccc(Br)c2)cc1', 'N[C@@H](c1c(F)cccc1F)[C@H]1CCO[C@]2(CCSC2)C1', 'Nc1[nH]ncc1CN(C1CCCCC1)[C@H]1CCSC1', 'NS(=O)(=O)/N=C(/[O-])c1ccccc1C1([NH+]2CCCCC2)CCSCC1', 'Oc1ccc(F)c(Sc2nc3c(cccc3c3C[NH+]4CCCCCC4)ccc23)c1', 'Clc1ccc(-c2ccccc2Cl)cc1', 'Clc1ccc(-c2ccccc2Cl)cc1', 'O=C(CS(=O)(=O)[N-]Cc1cccc(Cl)c1)NCCC(F)F', 'Clc1ccc(Br)cc1NCC[S@@](=O)Cc1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])c(F)cc(Oc2cncc([N+](=O)[O-])c2)c1C1CC1', 'Clc1ccc(COCOC2CCCC2)cc1', 'CN(C(=O)CSc1nnnn1C)c1cc(OC)ccc1OCC=C1CCCCC1', 'Oc1cc(O)c(Br)cc1OC[C@@H]1COc2ccccc2O1', 'O=C([O-])CN1N=N[C@@H](c2ccc([N+](=O)[O-])cc2)SC1=S', 'Oc1cccc2cc(OC3CCC[C@]23CO)c[nH]1', 'Clc1ccccc1[C@H]([NH3+])[C@@H]1C[NH+]2CCN1C[C@@H]2C', 'N=CCOC(=O)C1=c2ccccc2=[NH+][C@H]1Cn1ccnc1', 'ClCc1ccnnc1Cn1ccc2ccccc21', 'O=C([C@H](Cl)CS)c1cccc(F)c1', 'O=C(C/c1c(F)cccc1F)[C@@H]1Cc2ccccc2S1', 'NS(=O)(=O)c1cccc(NCC[NH+]2CCCCC2)c1', 'CNS(=O)(=O)[C@@H]1CCN(Cc2cccc(OC)c2)C1', 'OCc1ncc(CCOCc2ccccc2)s1', 'Oc1ccc(O[C@H](Oc2ccccc2)N2CCN(c3ccccc3)CC2)c(Cl)c1', 'Oc1ccc(F)cc1O/N=C/c1ccc(S(=O)(=O)[N-]c2cccc(Cl)c2)cc1', 'Clc1ccc(Br)cc1[C@@H]1CSCN1c1ccc(F)cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1Cc1ccccc1Br', 'OCS[C@H]1CCCO[C@]2(CC[NH2+]C[C@H]2c2ccccc2)[C@@H]2OC(O)=C2CC1', 'Clc1ccc(Br)c(F)c1-c1ccccc1', 'NS(=O)(=O)N[C@@H]1c2ccccc2C(=O)[C@@]12CCC[C@@H]2[NH+]C(=O)c1ccccc1Br', 'Clc1cc(CNC[C@H]2C[NH2+]CCO2)ncn1', 'Clc1cccc(-c2ncc(CSCc3n[nH]c(-c4ccccc4)n3)n2Cn2cncn2)c1', 'O=[N+]([O-])c1cc(N[C@@H]2CCN(C(=O)c3ccccc3F)C2)ccc1F', 'ClC(F)(F)OCCCNc1cc(Cl)ccn1', 'NS(=O)(=O)CC(=O)N(CC(=O)N1CCCCC1)c1cccc(Cl)c1', 'Clc1ccc(C2=NO[C@H](C[NH+]3CCCCC3)C2)cc1', 'Clc1cccc(NC2CCN(C(=O)C3=c4ccccc4=[NH+]C3)CC2)c1', 'CNC(=O)c1cc(S(=O)(=O)N2CCC[C@@H]2C2CCCCC2)cc2c1OCCO2', 'Oc1cccc2nc(CN3CCCC[C@H]3C(=O)[O-])nn12', 'Oc1cc(Br)c2[nH]nc(C(=O)/C=C/c3nnnn3-c3ccccc3)c2n1', 'Clc1cccc2c1N(Cc1cnn(Cc3ccccc3F)c1)CCC2', 'O=C([C@H](c1c(Cl)cccc1Cl)c1ccc(Cl)cc1)c1ccccc1[N+](=O)[O-]', 'Clc1cccc2[nH]nc(SC[C@@H]3CCCO3)c12', 'O=[N+]([O-])c1c(-c2cccs2)nc(-c2ccco2)n1Cc1cccs1', 'Clc1cccc2ccc(C[NH2+]C(c3ccccc3)c3cccs3)nc12', 'Clc1ccc(Cc2c[nH]nc2-c2ccccc2)cc1', 'C=C(C)Cn1c(C)c(C(C)=O)c2c1ncn2C', 'Clc1cccc([C@@H]2CCCN2C[C@@H](O)COc2cccc(Cl)c2)c1', 'Clc1cccc(Nc2cc(Br)ccc2OC[C@H]2CCCO2)c1', 'Clc1ccc(Br)cc1NCCCc1nnc2ccccn12', 'O=C(C1=NS(=O)(=O)c2ccccc21)N1CCN(S(=O)(=O)CC2CCC2)CC1', 'Clc1cc(CNC(=O)c2cc(C3CCCCC3)no2)cc([N+](=O)[O-])c1O', 'Clc1ccc(Br)cc1[C@@H]1Oc2ccccc2C(=O)N1CCCC(F)(F)F', 'Clc1ccsc1-c1ccncc1', 'Clc1ccccc1-c1nccs1', 'Clc1ccccc1-c1nccs1', 'Clc1ccccc1-c1nccs1', 'Clc1ccc(-c2cccs2)nc1', 'Clc1ccc(-c2cccs2)nc1', 'Oc1ccc(F)cc1Cn1cnc(C[NH2+]C2CCOCC2)n1', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n', 'Clc1ccsc1F\n\n\n', 'Clc1ccsc1F\n\n', 'Clc1ccsc1F\n', 'Clc1ccsc1F', 'Clc1ccsc1F', 'OC[C@@H]1N=N/C(=N\\CCCc2ccco2)C1', 'O=C(c1ccsc1)N1CCN(S(=O)(=O)OC[C@@H]2CCCO2)CC1', 'Oc1cccc(S[C@@H](C(=O)N2CCC(c3ccc(C(F)(F)F)cc3)CC2)c2ccccc2)c1', 'Nc1nc2ncc([N-]S(=O)(=O)c3ccc(Cl)cc3)cc2cc1C[NH2+]C1CC1', 'Clc1ccc([C@@H](Br)[C@H]2CC[C@@H]3CCC[C@H]23)cc1', 'Oc1cc(-n2nncc2SCc2cn(-c3ccccc3)nc2-c2ccc(F)cc2)ncc1Cl', 'O=C(COc1cc(F)ccc1N1CCCC1=O)c1ccccc1', 'Clc1ccc(Br)cc1Sc1nc(NC2CCCCCC2)n[nH]1', 'O=C(COc1ccccc1Cl)N1CCSCc2ccccc21', 'OCc1ncnn1C1CC1', 'Clc1ccccc1-c1ncc[nH]1', 'Clc1ccccc1-c1ncc[nH]1', 'Clc1ccc(-c2cc[nH]n2)cc1', 'Clc1ccc(-c2cc[nH]n2)cc1', 'Clc1ccc(-c2cc[nH]n2)cc1', 'Clc1ccc(Br)cc1-c1cnc2ccc(F)c(Br)c2c1N', 'Clc1ccccc1OCC(=O)Nc1ccc(C(=O)[O-])cc1', 'Clc1ccc(-c2n[nH]cc2CC2CC2)cc1', 'O=C(Cc1c(Cl)cccc1Cl)N(c1ccccn1)Cc1ccccc1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1OC1CC1', 'NS(=O)(=O)CC[NH+]1CCc2ccccc2C1', 'Clc1ccccc1NCC(C1CC1)S(=O)(=O)c1ccc(Cl)cc1', 'C=CCC[S@@](=O)Cc1nc2ccccc2n1C', 'Oc1cccc(NCC(=O)N2CCC[C@@H]2c2ccc(Cl)c(Cl)c2)c1', 'COc1nn(C)cc1CNC(=O)N(CCS(C)(=O)=O)c1ccccc1', 'O=C([C@H](NS(=O)(=O)c1cccc2ccccc12)[NH+]1CCCC1)c1ccc(F)cc1Cl', 'NS(=O)(=O)C[C@H](O)C1CCn2cccc2C1', 'CCc1ccc(-c2csc(=S)n2C2CCCCC2)cc1', 'Clc1ccc(Br)c(Cl)c1Cn1cccn1', 'Clc1cc(Br)cnc1Oc1ccccc1F', 'OC[C@@]1(COc2ccccc2C[NH2+]C(C)(C)C)CCCC1', 'Clc1cccc(S[C@H]2CCOc3ccccc32)c1', 'ClC(=S)c1ccnc(C2CC2)n1', 'Clc1ccc(Br)cc1OCC(=O)NC(C1CC1)C1CC1', 'Clc1cc(F)cc(NCC2(O)CC[NH+](CC[NH+]3CCCCC3)CC2)c1', 'Clc1cccc(F)c1-c1cccc(NCc2cccc(F)c2)c1', 'Clc1ccc(Br)cc1C[NH+]1CCC(N2CCOCC2)CC1', 'Clc1ccccc1[C@H](O)[C@@H]1CCO[C@]2(CCSC2)C1', 'O=C(CSCc1csc2ccccc12)Nc1ccc2[nH]c(=O)n(-c3ccccc3)c2c1', 'O=C(c1cn[nH]c1)c1ccnc(N2CCC(OC(F)(F)F)CC2)c1', 'Clc1ccc(-c2nnnn2C2CC2)cc1', 'Clc1ccc(-c2nnnn2C2CC2)cc1', 'Clc1ccccc1O/Cn1nccc1COc1ccc(Cl)cc1Cl', 'Clc1ccccc1Oc1cncc(-c2ccccc2)n1', 'Clc1ccccc1Cc1ncn(-c2ccccc2)n1', 'O=C(/N=c1\\o/c(=C\\c2ccc(F)cc2)[nH]c1=O)c1c(Cl)cccc1Cl', 'Clc1ccccc1Oc1cnc(N2CCOCC2)nc1', 'Clc1ccccc1OC1CCN(c2ncccn2)CC1', 'Clc1ccc(-c2nonc2N2CC(CO)C2)cc1', 'Clc1ccc([C@@H](Cl)/C=C/Nc2ccc(Cl)cc2)cc1', 'Oc1cc(OCC[NH+]2CCCCC2)n[nH]1', 'Clc1cc(Cl)nn1/N=N\\N1CCN(c2ncc(-c3ccccc3)s2)CC1', 'Clc1ccccc1OCc1ccn[nH]1', 'Clc1cccc([S@@]CCCCC(=O)Nc2ccc3c(c2)OCCC3)c1', 'Clc1cc(N2CCCCC2)c([N+](=O)[O-])cc1NCC1CCCC1', 'O=C(CNS(=O)(=O)c1ccc(F)cc1)NC1CCCC1', 'Clc1ccc(Br)cc1NC(=O)CN1CCCSc2ccccc21', 'O=C(/C=C\\Cc1ccc(Oc2ccccc2)c(F)c1)c1cc(O)ccc1F', 'Clc1ccc(-c2nnnn2/N=C/c2cccc([N+](=O)[O-])c2)cc1', 'Clc1ccc(-c2nnnn2/N=C/c2cccc([N+](=O)[O-])c2)cc1', 'Oc1ccc(Cn2cc([C@@H]3CC[NH2+]C3)nn2)cc1I', 'Clc1ccccc1OCc1csc2[nH]nc(-c3cccnc3)c12', 'COc1nn[nH]c1C(C)(C)OC(=O)Cn1c(=O)n(-c2ccc(C)c(C)c2)c(C)cc1=O', 'O=C([O-])[C@@H](Oc1ccccc1Br)n1cccn1', 'O=C(c1cnc(Nc2ccccc2)s1)N1CCC[C@H](c2cc[nH+]cc2)C1', 'O=C(c1c[nH]nc1-c1cccs1)N1CC[C@@H](Oc2ccccc2)C1', 'O=C(COCc1csc(Br)c1)OCc1ccc(F)cc1', 'Clc1cc(C[NH+](Cc2ccco2)Cc2ccccc2)ccc1Cl', 'ClC(=S)O[C@H]1CCCC[C@@H]1OCc1ccccn1', 'Clc1nnn(-c2ccccc2)c1-c1ccccc1', 'Clc1nncn1/c1ccccc1', 'Clc1nncn1-c1ccccc1', 'Clc1nncn1-c1ccccc1', 'CCOC(=O)N1CCC2(CC1)C[NH+]=C(N)N2c1ccccc1', 'N[C@@H](CCSCc1ccco1)C(=O)Nc1ccccc1', 'Clc1ccccc1Oc1cc(OCc2ccccc2)ncn1', 'Clc1ccc(Br)c(C[NH+]2CCN([C@@H](C(N)=O)c3ccccc3)C2)c1', 'Clc1ccc(CC[NH2+][C@H]2CCC[C@@H](O)C2)cc1', 'O=C(Cc1ccccc1)NC[C@@H](c1cccs1)[C@H]1CCCO1', 'Clc1ccc(C2(Sc3ncnc4ccccc34)CCC[C@@H]2CO)cc1', 'O=[N+]([O-])c1c(F)cccc1OCCN1CCN(C(=O)c2cccs2)CC1', 'NS(=O)(=O)C[C@@H](CNc1nc2ccc(Cl)cc2s1)N[C@@H]1CCS(=O)(=O)C1', 'Clc1ccccc1-c1n[nH]c2ccccc12', 'Clc1ccccc1-c1n[nH]c2ccccc12', 'Clc1ccc(-c2cc[nH]n2)c2ccccc12', 'OCc1n[nH]c(N2CCOCC2)n1', 'Clc1nncc(N[C@@H]2CCC[C@H](O)C2)n1', 'ClCc1nnn(C2CCCCC2)n1', 'COc1nc(C(=O)N2CC[C@]3(CCCC[C@H]3C)C2)no1', 'Clc1ccc(-c2cc(Sc3nc4ccccc4[nH]3)co2)cc1', 'Clc1ccc([C@@H]([NH2+][C@H]2CC[C@@H]([NH3+])C2)C2CC2)cc1', 'Clc1ccc([C@@H](Cl)[C@H]2C[C@@H]2CC(=O)[O-])cc1', 'Clc1cc(CN2CCN(Cc3ccccc3)CC2)cc(OCC(F)(F)F)c1', 'Clc1cccc(SCC/C=C/CO)n1', 'O=[N+]([O-])c1ccc(N[C@H]2CCC[C@H](S(=O)(=O)CCc3ccccc3)C2)cc1', 'Clc1ccc(-c2cc(O[C@H](C(=O)[O-])C3CC3)[nH]n2)cc1', 'O=C([C@H]1c2ccc(Br)cc2C1)N1CCC(NCc2cccc(F)c2)CC1', 'Clc1cc(Br)c2nc(-c3ccco3)cc(-n3ccnc3)c2c1', 'Nc1nnnn2c(C(=O)[O-])ccc12', 'Clc1cc(NSc2nnc(-c3ccc(Cl)cc3)n2Cc2ccc(F)cc2)ncn1', 'Oc1ccccc1CCn1nc(-c2cnccn2)ccc1=O', 'NS(=O)(=O)N(C)[C@H]1CCC[C@H]1c1cccc(Cl)c1', 'O=C(/N=C/NC[C@](O)(c1cccc(Br)c1)c1ccccc1)c1cccc([N+](=O)[O-])n1', 'CC(C)NC(=O)C(=O)Nc1cccc(SC(C)(C)C)c1', 'Clc1nncnc1NC1(c2ccccc2)CCOCC1', 'O=[N+]([O-])c1ccc(S(=O)(=O)NC[C@@H]2CCN(c3ccccc3)C2)cc1', 'Clc1cccc(N2CCC([NH+]3CCC[C@H]C3[C@@H]3CCCO3)C2)c1', 'NS(=O)(=O)c1ccc(S(=O)(=O)C[C@H](O)c2ccccc2)cc1', 'Clc1cc(Cl)cnc1-c1ccc(OC[C@H]2CCCO2)cc1', 'Clc1ccc(-c2cc(SC[C@H]3CCCO3)ncn2)cc1', 'NS(=O)(=O)Cc1noc(S(=O)(=O)NNC(=O)C(c2ccccc2)c2nc[nH]n2)n1', 'Clc1ccc([C@@H](Br)Cn2ccnc2)cc1', 'ClCc1cnc(Cl)n1Cc1ccccc1', 'COc1nc(-c2ccccc2F)cc(S(N)=[NH+]/CCNC(=O)c2ccc3c(c2)OCO3)c1Cl', 'Clc1cccc(NCCC[NH+](CCn2ccn[nH+]2)C2CC2)c1', 'Clc1ccccc1NCC[NH+]1CCC(COS(=O)(=O)c2ccccc2)CC1', 'Nc1nc(C[C@@H]([NH2+]Cc2cnco2)c2ccc(F)cc2Cl)ccn1', 'O=C([O-])[C@@H]1CC[C@@H](CNC(=O)Nc2ccccc2O)N1', 'O=C(/N=C\\c1ccco1)c1ccccc1[N-]Sc1ccccc1', 'O=C(c1c[nH]nc1-c1ccsc1)N1CCN(C(=O)[C@@H]2CCCc3ccccc3C2)CC1', 'COc1nnnn1/N=C/c1ccc(C)cc1', 'Oc1ccc(/C=N/N=C2\\C(=O)N(c3ccccc3)C(=O)NC2=S)cc1', 'O=C(c1cccn2cnnc12)N1CCC[C@H](Oc2ccc(Cl)cc2)C1', 'Clc1ccc(-c2n[nH]cc2N2CCN(c3ncccn3)CC2)cc1', 'Clc1ccc(CC(=O)[C@@H]2CSCN2C(F)(F)F)cc1', 'Clc1cccc2ncn(C[C@@H]3CCCCO3)c12', 'Oc1ccccc1[C@@H](NS(=O)(=O)c1ccc(F)cc1)c1cc(F)ccc1F', 'O=C(CNS(=O)(=O)C[C@@H]1CCCO1)Nc1ccccc1F', 'CNS(=O)(=O)CN(C)C(=O)C[C@@H]1COc2ccccc21', 'CN(Cc1ccc([O-])cc1)C(=O)c1ccc(Cl)c(N2CCCCC2)c1', 'Clc1ccc(Br)cc1CCCCBr', 'Clc1cccc2cc(-n3nnnc3NCCN3CCCCC3=O)ccc12', 'Oc1cc(F)ccc1N[C@@H]1CCO[C@]2(CCSC2)C1', 'O=C(COCc1ccccc1)Nc1cccc(OCc2ccccc2)c1', 'Nc1c(F)c([N+](=O)[O-])ccc1-n1cccc1', 'Clc1cccc2cccnc1n2CCNc1ccccc1', 'O=C(CNC(=O)COc1ccc(Cl)cc1F)NCc1cccs1', 'CNS(=O)(=O)c1cc(C)ccc1NC(=O)c1c(Cl)c(C)nn1C', 'Clc1ccc(Br)c(/C=C/C(=O)N2CCC[C@@H]2[C@@H]2CCCC2)c1', 'Clc1ccccc1OCc1ccn(-c2cccc(F)c2Cl)n1', 'Cc1cc([C@@H]2CCCCN2c2ncnc3[nH]ccc23)nc2ccccc12', 'O=[N+]([O-])c1c(Br)sc(CN2CCN(c3cccc(Cl)c3)CC2)c1CO', 'N#CCS[C@@H]1CCC[NH+](C2CCN(c3cc(Cl)ccc3)CC2)C1', 'Clc1ccccc1NS(=O)(=O)Nc1ccc2c(c1)CCC2', 'OCc1n[nH]cc1C[NH+]1CCC[C@H](Oc2cccc(Cl)c2)C1', 'ClCc1cc(OCc2cccc(Cl)c2)nn1CC1CC1', 'O[C@@H]1[C@@H]([C@@H](O)C[C@@H]2CCCOCC2)c2cc3ccccc3nc21', 'Clc1cccc(C2SCCCOC2)c1Oc1ccccc1', 'Clc1ccc(Br)c(Cl)c1C(=O)N1CCc2ncc(-c3cccc(Cl)c3)nc2C1', 'Cc1nonc1CCCN(C)C(=O)CCNc1ccccc1', 'COc1ncnc(NCc2ccccc2OCC2CC2)c1C', 'COc1nc(N[C@@H](COC(C)C)Cc2ccccn2)ccc1C', 'O=C(c1cncc(I)c1)N1CCC(OCc2ccccc2F)CC1', 'O=C(c1ccsc1CC1CC1)N1CCCc2ccccc21', 'O=C(/C(c1cccc(F)c1)NCc1ccccc1)[C@H]1CC=CCC1', 'Clc1nncn1c1c(C(F)(F)F)c(Cl)cn1C1CCC(O)CC1', 'Clc1ccccc1CSc1ccc(CO)cc1', 'O[C@@H]1c2cc(NC(=O)COCc3ccccc3)ccc2O[C@@H]1C(=O)N1CCCC1', 'O=[N+]([O-])c1c([N+](=O)[O-])c(F)cc(Nc2cc(Cl)cc(Cl)c2)c1Nc1ccc([N+](=O)[O-])cc1', 'Oc1cccc(-n2c([O-])c3n(c2=O)CCCCCCC3)c1', 'ClC[C@@H]1CN(c2ncccn2)C[C@H]1c1ccccc1F', 'O=C(CCC(=O)c1cccc(Cl)c1)N1CCC[C@@H]1c1ccc([N+](=O)[O-])cc1', 'Clc1cc(Cl)ncc1[C@@H]1CCOC2(CCCC2)C1', 'O=[N+]([O-])c1cc(N2CCC[C@@H]2c2nc3ccccc3s2)cc([N+](=O)[O-])c1', 'Clc1ccc(-c2cc(SC[C@@H](CCl)N3CCOCC3)ncn2)cc1', 'COCC[NH+](C)CCn1c(-c2ccc(F)cc2)nc2ccccc21', 'Nc1c(Cl)cccc1N[C@H]1CCN(Cc2ccccc2Cl)C1=O', 'ClCc1ccn(Cn2nnc(-c3cccc(Br)c3)n2)c1', 'O=C([C@H]1c2cc([N+](=O)[O-])ccc2O1)N1CCSC[C@H]1C1OCCO1', 'NS(=O)(=O)CCCNCc1c(Cl)cccc1Cl', 'Clc1ccccc1NS(=O)(=O)NC[C@H]1CCCCO1', 'Clc1ccccc1NS(=O)(=O)NC[C@@H]1CCCO1', 'Clc1ccccc1NS(=O)(=O)NCC[NH+]1CCCCCC1', 'O=C(COc1ccc2nccnc2c1)N1CC[C@H]([N+]2c3ccccc3CS2)C1', 'ClC[C@H]1O[C@]2(CCSc3ccccc32)O1', 'O=C(CS(=O)(=O)Cc1ccccc1)Nc1cc(Oc2ccccc2)c(S(N)=O)cc1', 'Clc1cc(C[NH2+][C@@H]2CSc3ccccc32)ccc1Br', 'COCCC(=O)N1CCc2ccc(-c3ncoc3C)cc21', 'Clc1cc(Cl)c(Cc2cccs2)c2ncc(-c3ccccc3)n12', 'O=C(c1cn[nH]n1)N1CCC(NC(=O)c2ccc(Cl)o2)CC1', 'ClCc1cn[nH]c1SCC[S@](=O)Cc1ccccc1', 'Clc1ccccc1Cn1nnc2cc(Cl)c(Cl)cc21', 'Oc1ccccc1O[C@@H](c1ccccc1)c1ncccc1O', 'Clc1ccc(-c2nnnn2-c2cccc(F)c2F)cc1', 'N[C@@H](CCc1nccs1)[C@H](c1ccccc1)[n+]1ccccc1', 'Clc1ccc(C[NH2+]Cc2cc(Br)ccc2O)cc1', 'COc1ncccc1F', 'ClCc1cnc2cc(Br)ccc2c1NC1CCCCC1', 'O=C(Cc1n[nH]c2ncccc12)N1CCC[C@H](c2nc(C(F)(F)F)cs2)C1', 'Clc1nnc([S-])=C(/C(=O)OCCN2CCOCC2)c1Cl', 'OC[C@H](CCS(=O)(=O)c1ccccc1)N[C@H]1CC(=O)N(c2ccccc2)C1', 'CNS(=O)(=O)c1ccccc1NC(=O)CCN1Cc2ccccc2OC1', 'Clc1cc(F)cc(-c2ccncc2)c1C[NH2+][C@H]1CC[NH+](Cc2ccccc2)C1', 'Clc1ccccc1Oc1cc(NC[C@H]2CCCO2)ncn1', 'Clc1ccccc1Nc1cc(OC[C@H]2CCCO2)ncn1', 'N[C@@H](Sc1ccccc1)c1ccccc1[N+](=O)[O-]', 'Clc1ccc(Br)cc1SC1CC[NH2+]CC1', 'Clc1cc(Cl)c(OCc2ncnn2Cc2ccccc2)c(C(F)(F)F)c1', 'O=C(c1cn[nH]c1)OCC(=O)Nc1ccc(F)cn1', 'Oc1cc(C[NH2+][C@@H]2CCCC[C@H]2CCO)cc(Cl)c1O', 'Oc1ccc(F)cc1C[NH+]1CCN(c2cc[nH+]cc2Cl)CC1', 'Clc1cccc2c(-c3cc([N+](=O)[O-])cc4ccccc34)ccnc12', 'Clc1ccc(CN2C[C@@H](c3n[nH]c(=O)[nH]3)CC2)c2ccccc12', 'Clc1cc(CN(Cc2ccccn2)C2CC2)ccn1', 'N#CCS(=O)(=O)c1ccccc1C[NH+]1CCc2ccccc2C1', 'Clc1ccc(Br)c(NC(=S)N(c2nccc([N+](=O)[O-])c2)[C@H]2CCSC2)c1', 'COc1nn(CC)c(CN2CC(=O)N(c3ccc(C)cc3C)C[C@@H]2c2ccccc2)c1C', 'N#C[C@@H](C(=O)c1ccc(CN2CCSCC2)cc1)c1ccoc1', 'Clc1ccc(-c2nn(-c3ccccc3)cc2C(F)(F)F)cc1', 'O=[N+]([O-])c1c(Br)c(I)c2c(n1)c1cc(-c3ccccc3)ccc12', 'O=C(CNS(=O)(=O)c1cc(Cl)ccc1CN1C(=O)c2ccccc2C1=O)c1ccccc1F', 'Clc1ccccc1NS(=O)(=O)N1CCN(Cc2ccc(F)cc2Cl)CC1', 'Clc1cccc(C(=O)OC2CC[NH+](Cc3ccccc3Cl)CC2)c1', 'Clc1ccc(Br)c(C[NH2+]C[C@@H]2CCN(c3cccc(F)c3)C2)c1', 'Clc1ccccc1Oc1cncc(Cn2ccc([N+](=O)[O-])n2)c1', 'Cc1cncnc1N/N=C1\\C(=N)N(C)C2=c3ccccc3=[NH+][C@H]21', 'CCn1nc(C)cc1CN1C[C@H]2CC[C@@H]1CC(CC(F)F)C2', 'Clc1ccc(Br)c(Cl)c1C[NH2+]Cc1cccc([N+](=O)[O-])c1', 'O[C@@H]1c2ccc3ccccc3c2CN1C(=O)CS[C@H]1CCS(=O)(=O)C1', 'Nc1[nH]ncc1S(=O)(=O)CCC(F)F', 'O=C(Cc1noc2ccccc12)Nc1ccccc1', 'Clc1ccsc1C1CC[NH+]([C@@H](C(=O)[O-])C(F)(F)F)CC1', 'Clc1ccccc1NC(=S)c1ccc(CCC(=O)[O-])cc1', 'Nc1n[nH]nc1CN(Cc1ccco1)[C@@H]1CCS(=O)(=O)C1', 'Oc1ccc([N+](=O)Cc2nc3ccccc3[nH]2)c2c1CCCC2', 'O=C(/N=C/c1ccc[nH]1)Nc1ccc(-c2ccc(F)cc2)nc1', 'O=[N+]([O-])c1c(OCO)sc2cc(O[C@H](CSc3ccccn3)[NH+]3CCCCC3)ccc12', 'Clc1ccccc1OCc1csc(N2CCCCCCCC2)n1', 'Oc1ccccc1-n1nnnc1COc1ccc2nccnc2c1', 'O=C(c1cc(Cl)cn1-c1nccc(C(F)(F)F)c1-c1ccccc1)c1cccc(C(F)(F)F)c1', 'CNC(=O)c1cc(CN2CC[NH+](C)C[C@H]2c2cccc(Cl)c2)no1', 'Oc1ccccc1[C@@H](Cl)C1CC1', 'Clc1ccc(Br)cc1-c1cnn([C@H]2CC[NH+](C3CCOCC3)C2)n1', 'Clc1ccc([C@@H]([O-])NC2CCCC2)cc1', 'CCS(=O)(=O)c1cc(C(=O)[C@@H](C#N)c2ccncc2)cc(OC)c1OC', 'O=C(c1cn2ccsc2n1)N1CCC(N2CCc3oncc3C2)CC1', 'Clc1cc(F)c(/C=N\\NC(=O)[C@H]2CSCCS2)cc1Cl', 'Cc1nonc1C[C@H](CS)Cc1ccccc1', 'ClCc1csc([N-]Cc2nc(-c3ccccc3)no2)n1', 'OC[C@@]1(Cc2cccnc2)Oc2ccccc2C1', 'Clc1cc(Cl)nn1C[C@@H](O)COc1ccccc1', 'ClCc1cc(-n2cccc2)cc2c1NCCCO2', 'Nc1[nH]nc(S[C@@H]2CCOC2)c1[N+](=O)[O-]', 'O=C(CNCc1csc(-c2ccccc2)n1)c1ccccc1O', 'Clc1cccc([S@@]c2c[nH]nc2)c1', 'Clc1ccsc1Sc1cc([N+](=O)[O-])ccc1N1CCCCCC1', 'O=C(c1cnc([C@@H]2CCCC[C@@H]2C(=O)N2CCCC2)s1)N1CCOCC1', 'ClCc1csc2nc(N3CCC[C@@H](C(=O)N4CCC[C@H]4C)C3)nn12', 'O=C(COc1cc(Br)ccc1OCc1ccccc1)N(Cc1ccc(F)cc1)S(=O)(=O)c1ccco1', 'Oc1cccc(NC(=O)CCN2C(=O)[C@H](COc3cccc(Br)c3)C2=O)c1', 'O=C(/C=C[C@@H]1CCCS1(=O)=O)c1cccc2ccccc12', 'Clc1ccc(Br)cc1-c1cnn(CCc2ccccc2)n1', 'C=C[C@@H](SC)c1nc(-c2cc(F)ccc2F)n[nH]1', 'Clc1ccc(Cc2noc(Cc3cccs3)n2)cn1', 'Oc1cc(OCOC[C@@]2(CBr)C3=CCC[C@@H]2C3)ccc1F', 'O=C([O-])[C@H](CCc1cc(Br)ccc1F)N1C[C@@H]2CC=CC[C@@H]2C1', 'Clc1ccc(Br)cc1CCNc1ccccn1', 'COc1nn(-c2c(C)cccc2F)c2cccnc12', 'Clc1cc(NSc2nc3ccccc3[nH]c2=O)n(-c2ccccc2)n1', 'Clc1ccccc1Nc1cnc(-c2ccccc2)o1', 'CNS(=O)(=O)[C@@H]1CCN(C(=O)c2ccccc2)C1', 'Clc1nnc(S[C@@H]2CCC[C@@H](O/N=C/c3ccccc3)C2)n1C1CC1', 'Clc1ccc([C@@H](O)c2ccc(Br)cc2)c(F)c1', 'Clc1ccccc1-c1cscc1C[NH2+]c1ccccn1', 'O=C(Cc1coc([C@@H]2CCCO2)c1)Nc1cccc2ccccc12', 'Clc1nncc(-n2cccc2)c1/C=C/C(=O)N1CCc2ccccc21', 'Clc1ccsc1Sc1cccc(F)c1', 'Clc1cccc2[nH]nc(S(=O)(=O)[N-]c3ccc(-n4cccn4)c([N+](=O)[O-])c3)c12', 'Clc1ccccc1-c1n[nH]cc1C(=O)CN1C(=O)COc2ccc(F)cc21', 'Oc1cc(F)ccc1-c1cccc(S(=O)(=O)N2CCN(c3ccccc3)CC2)c1', 'O[C@@H]([C@@H]([NH2+]Cc1ccccc1)Sc1ccccc1)c1cccc(F)c1', 'ClCc1csc2ncn(Cc3ccc(Oc4ccccc4)cc3)c12', 'Clc1ccccc1-c1nc(N2CCCCCCC2)co1', 'Clc1cccc2nc(/C=C/C3CCCCC3)nn12', 'O=C(CCCSC1CCCC1)NCc1nc2ccccc2[nH]1', 'Clc1ccccc1[C@H]1NN[C@H](c2ccco2)N1O', 'COc1nnnc(N2CCc3nn(-c4ccc(F)cc4)n(C)c3c2C(=O)OC)c1C', 'O=[N+]([O-])c1ccnc2c1OCC[C@H]2NC(=O)[C@H]1CCCc2ccccc21', 'Oc1ccc(Oc2cc([C@@H](O)C[NH+]3CCCC3)ncn2)cc1F', 'O=C(Cc1c[nH]nc1-c1ccccn1)N[C@H](CO)c1ccccc1', 'O=C([O-])c1csc(C[NH2+][C@H](c2ccccc2)C2CC2)n1', 'Clc1nnn(-c2ccccc2)c1N[C@H]1CCOc2ccccc21', 'Clc1ccccc1-c1nc(C[C@@H]2COc3ccccc3O2)no1', 'Clc1ccc(Cc2noc([C@H]3COc4ccccc4O3)n2)cc1', 'Clc1cccc(-c2ccc(C[NH+]3CCCC3)o2)c1', 'Clc1cc(C2COC2)ccc1OCc1ccc(Br)o1', 'Clc1cc(Cl)ccc1CN1CCCCC[C@@H]1c1ccccn1', 'O=C(CCSCc1[nH+]ccn1CC(=O)[O-])c1ccccc1', 'Oc1cccc(F)c1SCC(=O)N1CSC[C@@H]1c1ccccc1', 'Clc1ccc(Br)c([C@@H]2[C@H](C(=O)[O-])CCO2)n1', 'Clc1cccc(F)c1Cn1ccc(-c2ccc(Cl)cc2)n1', 'Clc1ccc(Br)cc1/N=N\\[C@@H]1CCCC[C@@H]1S(C)(=O)=O', 'N=C1CSc2ccc(C(=O)NC3CCCC3)cc2N1', 'Oc1cc(COc2n[nH]c(=O)[nH]2)ccc1C[NH2+]Cc1ccco1', 'CN(C)/C=C/C(=O)Nc1ccccc1OCc1cccc(Cl)c1', 'Clc1ccc(Br)cc1N(Cc1ccc(Br)o1)C1CC1', 'OC[C@@]1(OC2CC2)CCC[C@@H]1c1ccccc1F', 'O=C(Cc1c(O)cccc1Cl)N1CCC[C@H](O)C1', 'Clc1ccc(Br)c(Cn2ccc3ccc(O)cc32)c1', 'Oc1ccccc1OSCOc1ccc(Br)cc1', 'CNS(=O)(=O)[C@@H](c1ccccc1)c1ccccc1Br', 'Oc1cc(F)ncc1S(=O)(=O)N[C@@H]1CCN(c2ccc(F)cc2)C1', 'Nc1nonc1-c1ccccc1', 'Nc1nonc1-c1ccccc1', 'Nc1nncn1-c1ccccc1', 'COc1nc(-n2nc(C)cc2CC(=O)N2CCCc3cc(C)ccc32)cs1', 'Oc1cc(O[C@@H](c2nc3ccccc3s2)C[C@H]2CCCO2)cc(O)c1', 'Clc1ccc([C@@H](Br)C[C@@H]2CCCO2)cc1', 'Clc1cc(/C=C/Cc2ccccc2F)ccc1Cl', 'Clc1ccc([C@@H]([C@@H](C(=O)[O-])Cn2cccc2)C(F)(F)F)cc1Cl', 'O=C(c1cnn(CCO)c1)[NH+]1CCC[C@@H]1CC#N', 'Clc1ccc(Br)cc1CC[NH2+]C1(CO)CCCCC1', 'O=C(c1cnc(SCc2ccc(F)cc2)s1)N1CC[C@H]2CCCC[C@@H]21', 'O=C([C@H](Sc1ncccn1)C(=O)NC1CC1)NCc1cc(F)ccc1F', 'O=C(CNC(=O)C[NH+]1CCN(c2c(F)cccc2F)CC1)Nc1cccc(Cl)c1', 'Clc1ccc(Br)cc1OS(=O)(=O)CCO', 'O=C(Cc1cc(Cl)ccc1F)Nc1ccccc1-c1ncccn1', 'O=[N+]([O-])c1cc(Br)c(S(=O)(=O)Nc2ccc(F)cc2)c2c1CCO2', 'Clc1cc(/C=N/O[C@H]2CC=CCC2)ccc1Cl', 'Clc1ccc([C@@H](N[C@@H](CO)c2ccc(Cl)cc2)[NH+]2CCCC2)c(F)c1', 'Clc1ccc(Cc2noc(C3CC3)n2)cc1', 'Oc1ccc([C@H]2C=C(c3ccc(F)cc3)Nc3ncnn32)cc1', 'Clc1ccccc1[C@H]([NH2+]/N=C/c1cccc(Cl)c1)n1cnnn1', 'O=C([O-])/C(=C\\c1ccc2ccccc2c1)C1CCCCC1', 'O=C(c1ccsc1)c1ccccc1COC1CCCCC1', 'NS(=O)(=O)CC[C@H](c1ccccc1)c1n[nH]cc1C(C)=O', 'O=C(CCS(=O)(=O)c1cccc2ccccc12)OCc1ccccc1', 'Clc1ccccc1Cc1nc(-c2ccccc2Cl)no1', 'Clc1ccccc1-c1noc(Cc2ccccc2F)n1', 'Clc1cc(COC2CC2)on1', 'Nc1[nH]ncc1N[C@@H]1CCN(c2ccccc2)C1=O', 'O=[N+]([O-])c1cc(Cl)ncc1[C@@H]1CCOC2(CCCCC2)C1', 'Clc1cc(Cl)nnc1N[C@@H]1CCOC2(CCCCC2)C1', 'Clc1ccc(Br)cc1NCC1(C2CC2)CC1', 'Oc1cc(O[C@@H]2CCN(c3cccc(Cl)c3)S2)cc2c1OCCO2', 'Oc1cc(F)nc(S[C@H]2CC[NH2+]CC2)c1N1CCCC1', 'N[C@@H](S/N=C(\\C)c1ccco1)N1CC=C(Cc2ccccc2)C1=O', 'Clc1nnnn1/N=C/c1ccc(Br)cc1', 'O=C(c1cncc(I)c1)[C@@H]1CN2CCC[C@@H]2CO1', 'O=C(COCCCc1cccc(Cl)c1)N[C@@H]1CCc2c(O)cccc21', 'O=C(CN(C(=O)c1cccc(Cl)c1)N1CCCC1)c1ccccc1', 'Clc1ccccc1O/N=C(/[O-])Cc1cccc(OC2CCCC2)c1', 'Clc1cccc(/C([O-])/N=C(\\[O-])CC[C@@H](C(=O)[O-])c2ccccc2)c1', 'Clc1ccc(Br)c(Oc2cccc(OC[C@H]3CCC(=O)N3)c2)c1', 'Oc1cc(O)cc(Cl)c1C[NH2+]Cc1ccc(Cl)nc1', 'Cc1c(Br)cccc1NC(=O)NCC(C)C', 'Oc1cc(Cn2ncc3cc(Cl)ccc32)ccc1O', 'Clc1cc(C(CO)OCc2ccncc2)n[nH]1', 'ClCc1csc(C[C@@H]2CCCOC2)n1', 'ClC(F)(C(F)F)Nc1ccccc1C(F)(F)F', 'O=C(c1c[nH]cn1)O[C@@H]1CCS(=O)(=O)C1', 'Clc1ccc(-c2nnnn2Cl)cn1', 'Clc1ccc(-c2nnnn2Cl)cn1', 'Clc1ccc(C2(NCC(=O)[O-])C[C@@H]2C2CCOCC2)cc1', 'Oc1cc(Cn2nc(-c3ccc(Br)cc3)ccc2=O)ccc1Cl', 'O=C(C[NH+]1CCCC1)NC[C@@H](O)c1ccc(Cl)cc1', 'Clc1cncc(C2CC2)c1', 'Clc1nnc(Cl)nc1C1CC1', 'ClCc1ncn2cccc(-c3ccncc3)c12', 'Clc1nnnn1CC(=O)N(Cc1ccccc1)[C@H]1CCS(=O)(=O)C1', 'Clc1ccccc1O/C=CCN1C(=O)CN(c2ccc(Cl)cc2)C1=O', 'O[C@@H]([C@@H]1C[C@H]1c1ccc(F)cc1)c1ccc(F)cn1', 'O=C(c1cn[nH]c1)[C@@H]1C[C@@H]1c1ccc(Br)cc1', 'Clc1ccccc1Nc1cc(I)cc(CN2CCOCC2)n1', 'O=C(COc1ccc2ccccc2c1)NCc1ccc(Cl)s1', 'Clc1ccccc1Nc1nncc(-c2ccccc2Br)n1', 'Clc1nnn(Cc2ncnn2C2CCCCCC2)c1Cl', 'Clc1cccc(S[C@H]2OCc3ccccc32)c1[N+](=O)[O-]', 'Clc1ccc(Br)cc1-n1nncc1C[NH+]1CCCCC1', 'OCc1nnnn1[C@H]1CCCN(c2cc(F)cc(F)c2F)C1=O', 'NS(=O)(=O)[C@@H]1CCCN(C(=O)CSc2cccc([N+](=O)[O-])c2)C1', 'ClC(c1ccccc1)[C@@H]1CCS(=O)(=O)C1', 'COC(=O)[C@H](C[NH+]1CCCCC1)c1ccc(F)cc1', 'O=C([O-])[C@@H]1CSCN1C(=O)Cc1ccccc1Cl', 'Clc1cc(N2CCCc3ccccc32)ncn1', 'O[C@@H](Sc1ccc(F)cc1Cl)Cn1ccnc1', 'OCc1n[nH]cc1CNCc1ccc([N+](=O)[O-])cc1', 'Nc1c(Cl)c([O-])c2cc(F)ccc2c1S(=O)(=O)/N=C(\\[O-])C12CC3CC(CC(C3)C1)C2', 'COc1nn(CC2CCCC2)c(C)c1S(=O)(=O)N(C)CCC(=O)C(F)(F)F', 'Clc1ccsc1C(=O)N[C@@H](c1ccccc1)c1ccccc1O', 'Clc1ccccc1-c1cscc1C(=O)Nc1ccc(C(=O)[O-])cc1', 'O=C(c1c[nH]c(N2CCCCC2)n1)N1CC[C@H](n2cncn2)[C@@H]1C#N', 'O=C(c1cccs1)N1CCC[C@@H](C(=O)OCc2ccc(C(F)(F)F)cn2)C1', 'O=C(CNC(=O)Nc1nc2c(s1)CC[C@@H](C(=O)[O-])O2)c1ccccc1', 'Clc1ccccc1Cn1nc2cc(-c3ccccn3)[nH]nc2c1=O', 'Oc1ccc(/C=N/[NH+]C(=O)COc2ccccc2Br)cc1Br', 'Clc1cccc2c1O[C@@H](c1ccccc1Br)[C@@H]1C[C@@H]1[C@@H]1CCCN21', 'NS(=O)(=O)Nc1cc([N-]S(=O)(=O)c2cc(F)ccc2F)ccc1F', 'Oc1cc(-n2cnc([C@@H]3CCCN(c4ccccc4OC(F)F)c3)ncn2)ccc1O', 'O=C(/N=c1/oc2ccccc2c2ccccc12)c1ccc(NC(=O)C(F)(F)F)cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1[C@H]1Cc2ccccc2O1', 'Cc1cnc2c(c1)[nH]c(=N)n2C[C@@H]1CCCO1', 'COC[C@H]([NH3+])Cc1ccc(OC(F)F)cc1', 'O=C([C@H](Cl)CBr)c1ccccc1', 'O=C(c1cc(-c2ccccc2)n[nH]1)N1CCN(S(=O)(=O)c2ccccc2Cl)CC1', 'Clc1cccc(NC2(c3cccs3)CC2)c1F', 'Oc1ccc(/C=N/N[C@@H]2CCSC2)[nH]1', 'O=C(/C(Cc1ccccc1)Nc1nccs1)c1ccc(F)cc1', 'Clc1ccc(Br)c([C@@H]2SCCC[C@@H]2N2CCOCC2)c1', 'Clc1cc(/C=N/N=C\\c2ncn(-c3ccccc3)n2)c[nH]1', 'Clc1ccccc1Cc1nnc([C@@H]2CCCC[NH+]2Cc2cccc(C(F)(F)F)c2)o1', 'Clc1ccc([C@@H]([NH2+][C@H]2CCc3ccc(F)cc32)[S@@](=O)Cc2ccccc2)nc1', 'Clc1ccsc1[C@@H](CO)CC[C@H]1CCCCO1', 'Clc1cc(Cl)nnc1N/C=C/c1ccc(CCCO)o1', 'Clc1cc(NC[C@@H](c2ccccc2)n2cc(S(=O)(=O)N3CCCCC3)cc2)ccc1Cl', 'NS(=O)(=O)N[C@@H]1CCN(c2ccccc2F)C1', 'Clc1ccc(Br)cc1[C@@H]1COCC[NH+](Cc2ccco2)C1', 'O=C(C[NH+]1CCC[NH+](Cc2cccc(Cl)c2)CC1)Nc1ccccc1F', 'Clc1cc(/C=C/Nc2ccc(OCc3ccccc3)cc2)ccc1Cl', 'Oc1cccc(-c2cnn(-c3ccc([N+](=O)[O-])c(NC4CCCCC4)c3)n2)c1', 'Oc1ccccc1CN[C@H](CO)c1cc([N+](=O)c2cccnc2)ccc1F', 'Clc1ccc([C@@H](CC2COc3ccccc3CC2)NCCc2cccnc2)s1', 'Clc1ccc(-c2n[nH]c3[nH]c[n+]([O-])c23)cc1', 'O=C(CS(=O)(=O)[N-]c1ccccc1C(=O)c1ccc(F)cc1)NNC(=O)c1cccs1', 'O=C(c1cn2ccsc2c1NS(=O)(=O)c1ccccc1)N1CCC[C@@H]1c1cccc(Cl)c1', 'Clc1ccc(Br)cc1CCNC(=O)c1cccs1', 'Clc1ccc(Br)cc1-c1cnn([C@@H]2CCC[NH2+]C2)n1', 'Clc1ccccc1[C@H](CCO)Nc1ccc2nccnc2c1', 'Clc1ccccc1Oc1cc(CCCn2cccn2)ncn1', 'Cc1cc(F)ccc1NC(=O)[C@@H]1CCCN(C(=O)C(=O)NCc2ccc[nH]2)C1', 'Oc1cc(-n2cc(Br)cn2)ccc1NCc1ccc(F)cc1', 'O=C(c1cc(Br)c2c(c1)OCO2)N1CCC(Oc2ccccc2)CC1', 'Clc1ccccc1-c1cn2cc(Br)cnc2c1[O-]', 'Clc1ccc(Br)cc1[C@@H]1COCC[NH+]1C[C@@H]1CCCO1', 'CNc1[nH+]/c(=C\\c2ccccc2Cl)nc([O-])c(OC)c1OC', 'ClCc1csc(CN2CCN(c3ccc(Oc4ccccc4)nn3)CC2)c1', 'Clc1ccsc1[C@@H]1CC(C(=O)N2CCN(CCO)CC2)=NO1', 'O[C@@H]1c2cc(-c3ccco3)ccc2O[C@@H]1c1cccc(Br)c1', 'O=C(/N=C/N1CCN(Cc2ccco2)CC1)C(F)(F)F', 'N[C@@H](C(=O)Nc1ccccc1C(F)(F)F)[C@H]1CCCO1', 'Oc1cc(-n2cnc(Cl)n2)ccc1OCCN1CC[NH2+]CC1', 'Clc1cc(NC[C@@H](n2cncn2)[NH+]2CCCC2)ccc1F', 'Clc1ccccc1Cc1n[nH]c(Nc2cc(Cl)ccc2Cl)n1', 'ClCc1cnc(COCCCOC2CCCCCC2)s1', 'ClC[C@H]1CCCN(Cc2csc(-c3ccccc3)n2)C1', 'Clc1cccc2nc(C[NH+](Cc3ccco3)[C@@H]3CCS(=O)(=O)C3)ccc12', 'O=C(COc1cc(Br)c2ccccc2n1)Nc1ccccc1', 'Clc1ccccc1Oc1cc(COc2ccc(Br)cc2)ccn1', 'Oc1cccc(C2=NN([C@H]3[C@@H](C(=O)[O-])C[C@H](O)N(c4ccccc4)[C@H]3c3ccccc3)C2)c1', 'Clc1nnn(-c2ccc(F)cc2)c1CN1CCCCCC1=O', 'O=C(c1cncc(-c2ccccc2)n1)N1CC[C@@H](N2CCCC2=O)C1', 'ClCc1nonc1NC(=O)Cc1csc(N2CCCC2)n1', 'ClCc1nnsc1NCc1ccnn1C1CCOCC1', 'Clc1ccsc1-c1nnc(C[C@@H]2C[C@H]2c2ccccc2F)o1', 'ClCc1csc([C@H]2O[C@@H]3CCN(c4ccc(F)cc4)CCC[NH2+]CCC[C@@H](O)[C@H]32)n1', 'COC(=O)Nc1ccc(O[C@@H](C(C)C)[NH+](C)C)cc1', 'Clc1ccc(-c2ccccc2)cc1NC[C@H]1CCOC1', 'O=C(c1cccs1)[C@H]1CCCN(c2ccccc2)C1', 'Clc1ccccc1OCCCNCc1csnn1', 'Clc1cc(C[NH+]2CCC[C@@H]2CO)ccc1O', 'Clc1ccc([C@@H](Cl)CC[C@@H]2CCCCN(S(=O)(=O)c3ccccc3)C2)cc1', 'Clc1ccsc1-c1ccoc1C[NH+]1CCCC1', 'Clc1ccc(-c2ccccc2)cc1C(=O)OCC(=O)Nc1cc(F)ccc1F', 'Clc1ccsc1[C@H]([O-])[C@@](C)(CO)c1ccccc1', 'O=C(c1c[nH]nc1C(F)(F)F)N1CCC[C@@H]1CO', 'Clc1ccc(Br)cc1[C@@H]1Cc2ccccc2S1', 'Oc1ccc(F)cc1-c1nnc(CNCc2ccc(C(F)(F)F)cc2F)o1', 'Oc1cc(-n2cnnn2)ccc1N[C@@H](CO)Cc1ccccc1', 'ClC[C@H]1O[C@@](NC(=O)C2CCC2)Cc2ccc(Br)cc21', 'Clc1ccccc1-c1nn(-c2ccc(Br)cc2)cc1[O-]', 'COc1nn[nH]c1/C(C)CCc1cccc(F)c1', 'Clc1ccccc1Nc1cnc(S(=O)(=O)N(C(C)C)N2CCOCC2)c2ccccc12', 'Oc1cc(/C=N/NC(=O)C[C@@H](c2cccs2)N2CCOCC2)nc2ccccc12', 'Nc1cn[nH]c1C(=O)CCCCS(=O)(=O)c1ccc(F)cc1', 'Clc1cc(F)c(C2C[C@@H](CO)CC2)nc1S(=O)(=O)N1CCCCC1', 'COc1ncc(Cl)c([C@@]2(CC)COc3ccccc32)c1[N+](=O)[O-]', 'Clc1cccc(Cn2c3ccccc3nc2SCc2ccc(Br)cc2)c1', 'Clc1cccc(NC[C@@H]2CC[NH+](CN3CCC(O)CC3)C2)c1', 'Oc1ccccc1[C@H](O)[C@@H]1CCc2ccccc2C1', 'Oc1ccccc1-c1c(O)ccc2c1CCCC2', 'ClCc1nnc([C@]2(CC(F)(F)F)N=CN2C(=O)c2ccsc2)s1', 'Clc1ccc(CNS(=O)(=O)c2ccc(Br)cc2F)cc1', 'O=C(CS(=O)(=O)CCN1C(=O)/C(=C/c2ccc(C#N)cc2)SC1=S)Nc1ccccc1', 'Clc1ccccc1OCc1csc(OC2CC2)n1', 'C[C@@H]([NH2+][C@@H]1CCN(CCn2cc[nH+]c2C(C)(C)C)C1)c1nccs1', 'ClC[C@H]1CN(c2ccccc2)CC1', 'ClC[C@H]1CCCN(c2ccccc2)C1', 'Clc1ccccc1NS(=O)(=O)NC1CC1', 'Clc1nnc(/C=C/CCO)c([O-])n1', 'C=C[C@@H]1Cn2c(nnc2S[C@H](C)C(=O)N(CC)Cc2ccccc2)C1', 'O=C(c1cn[nH]n1)N(Cc1ccsc1)c1nc(C(F)(F)F)cs1', 'ClC(=S)[C@@H](NC(=S)Nc1cc(F)ccc1F)c1ccc(Cl)c(Cl)c1', 'ClC(=S)O[C@@H](c1ccc(Cl)cc1)c1ccccc1F', 'Clc1nnc(S[C@H]2CC(=O)N(c3ccc(F)cc3)C2)n1Cc1ccco1', 'Oc1cccc(NC2(c3ccccc3)CC2)c1', 'Nc1[nH]nc2c1CCCC2', 'O=C(c1cnn(C(=O)N[C@@H]2CS(=O)(=O)C[C@@H]2C(=O)[O-])c1)c1ccccc1', 'O[C@@H]1C(C(=O)OCc2ccccc2)c2ccccc2S1', 'Clc1ccc(-c2csc3sccc3c2=O)cc1', 'Clc1ccccc1CS(=O)(=O)C1CC[NH+](C2CCCCC2)CC1', 'O=[N+]([O-])c1cc(OCc2ccccc2)c(Cl)cc1OCO', 'Clc1ccc(CNS(=O)(=O)c2ccc([N+](=O)[O-])cc2)cc1', 'O=C([O-])/C=C/c1cccc(O[C@H]2CCN(c3ccccc3)C2=O)c1', 'O=C([C@H]1CC[C@H](C[NH3+])O1)c1ccc(Br)cc1', 'N#C[C@@]1([NH+]2CCC(n3cncn3)CC2)CCCSC1', 'Clc1ccc(Br)cc1S[C@H]1C[C@@H](O)CN1C(=O)C1CC1', 'Oc1ccc(O[C@@H](C(=O)[O-])c2ccccc2)cc1', 'Oc1cc(F)nc(N2CCCCCCC2)c1', 'Clc1cccc(Br)c1Sc1cccc(Br)c1', 'Oc1cc(O[C@@H](CC(F)F)c2c[nH]nc2-c2ccccc2)ccc1O', 'Oc1cc(CC[NH2+][C@H](c2ccccn2)c2ccco2)ccc1F', 'Clc1nnnn1c1[nH]c(C2CC2)nc1N1CCc2ccccc21', 'Clc1ccc(CCOC2CCOCC2)cc1', 'Oc1cc(OCCOCOc2ccccc2)ccn1', 'Clc1ccccc1-c1csc(C[NH2+]CCO)n1', 'Cc1sn(-c2ccc(F)cc2)c(=O)c1CCC(=O)Nc1ncnc2sccc12', 'Clc1cccc(Br)c1O[P@](=O)(=O)c1ccccc1', 'Clc1cccc(Br)c1CO/N=C1\\C(=O)Nc2ccccc21', 'Clc1ccc(-c2n[nH]c(NC3CCCC3)n2)cc1', 'Clc1nnnn1CC1CCC1', 'Clc1nnnn1C1CC1', 'Clc1nncc(OC2CCCC2)n1', 'Clc1nnnn1CCSc1nc(CC2CC2)n[nH]1', 'ClCc1ccc2nsnc2c1C[NH+]1CCCC1', 'Clc1ccccc1O/N=[N+]=SP1CC1', 'O=C(CNc1ccccc1[N+](=O)[O-])c1ccccc1', 'O=C(/C=C\\Nc1ccccc1[N+](=O)[O-])c1ccccc1F', 'Clc1ccccc1-c1nnc(C[NH+]2CCN(Sc3ccccc3)CC2)c(=O)[nH]1', 'Clc1ccccc1Nc1cc(OCc2nc3ccccc3s2)cs1', 'O=C(CCc1cc[nH]n1)Nc1ccc(Cl)cc1O', 'Cc1sc(=O)n(C)c1N1CCC(C(=O)NCCc2ccc(F)cc2)CC1', 'Clc1ccc(-c2nn([C@@H]3CC[C@@H](O)[C@@H](CO)[C@@H]3O)cc2F)cc1', 'O=C(Cc1nc2ccccc2c(=O)[nH]1)Nc1ccc2ccccc2c1', 'O=C(CNC(=O)c1ccc2c(c1)N(C(C)C)CCC2)NCC(=O)[O-]', 'N=CCNS(=O)(=O)c1cccc(C(=O)N[C@@H]2CCCC[C@@H]2C)c1', 'C[C@@H](C[NH+](C)CC(=O)NCc1cccc(F)c1)NS(C)(=O)=O', 'Oc1cc(-n2cnnn2)ccc1Nc1ncc([N+](=O)[O-])cc1Br', 'Clc1ccc(-c2nnn(-c3ccccc3)n2)cc1O', 'O=C([O-])/C=C(/C(=O)NCc1noc(-c2ccc(F)c(F)c2)n1)c1ccccc1O', 'Clc1ccccc1NS[C@H](C(=O)[O-])C(=O)Nc1ccccc1', 'O=[N+]([O-])c1ccc2oc(C(=O)N3CCN(c4ccccc4[N+](=O)[O-])CC3)nc2c1', 'Oc1cc(OC2CC[NH2+]CC2)ccc1Oc1cccc(Br)c1Br', 'Clc1ccc(CO)c([C@@H]2CSCC[NH2+]2)c1', 'Oc1cc(C=NOc2[nH]nc(Nc3ccccc3)n2)ccc1O', 'Clc1cccc(Br)c1OCCc1ccccc1', 'Clc1cc(C[NH+]2CCC[C@@H]2c2ccc[nH]2)ccc1F', 'Clc1ccccc1-c1csc(NC[C@@H]2CN(Cc3cccnc3)C[NH+]2Cc2ccccc2)c1', 'Oc1cccc(SC2(C(=O)c3ccccc3)CCOCC2)c1', 'NS(=O)(=O)N(CCc1ccccc1)c1nc2ccc(Br)cn2n1', 'O=C(/C(c1cccs1)(C(=O)[O-])C1CCN(S(=O)(=O)c2ccc([N+](=O)[O-])cc2)CC1)/c1ccccc1', 'COc1nn(CC)c2ncc(C(=O)c3cc4ccccc4o3)nc12', 'Clc1ccc([C@@H]([C@@H]O)N[C@@H]2CCC[C@@H](O)C2)cc1', 'Oc1cccc2cc(C[NH2+][C@@H]3CC[NH+]c4ccc(Cl)cc43)oc12', 'Clc1nncc(S[C@@H]2CC[C@@H](COc3cc([N+](=O)[O-])ccc3F)C2)n1', 'N=CCn1cccn1', 'CN=N/NC(=O)[C@H](C)S(=O)(=O)C[C@@H](C)c1ccccc1', 'Clc1cc(Br)c(C[NH+]2CCN([C@@H]3CCC[C@H]3O)CC2)cn1', 'O=C([O-])[C@H](Oc1ccccc1F)C(F)(F)C(F)(F)F', 'N[C@@H]1c2ccccc2[C@@H]2OCC[C@H]21', 'Clc1ccccc1O/Cn1cnn[nH+]1', 'CNS(=O)(=O)c1ccc(=O)n(CCC(=O)N(C)Cc2ccc(Cl)cc2)c1', 'O=C(/C(c1cccs1)[C@@H]1CC=CC=C1O)c1ccccc1O', 'Clc1cccc(F)c1C[C@@H](C[NH3+])c1c(F)cccc1F', 'CCS(=O)(=O)Cc1ccccc1OC[C@@H]1Cc2ccccc2O1', 'Clc1cccc(C[NH2+]Cc2nc(C3CC3)ns2)c1', 'Oc1ccc(/C=N/NC(=O)c2ccc(-c3ccccn3)s2)cc1', 'N[C@@H](O)c1cccc(NC(=O)N[C@H](CSC(F)F)c2cccc(F)c2)c1', 'Clc1cc(Cl)nn1CCN1CCOCC1', 'Oc1cc(OCOC2CCCC2)c(Br)c(Cl)c1OO', 'O=C(COCCC[NH+]1CCCC1)Nc1cc2c(cc1Cl)OCCO2', 'Clc1cc(NC2CC[NH+](Cc3ccc4c(c3)OCCO4)CC2)ccc1F', 'Clc1ccccc1-c1noc(C[NH+]2[C@@H](C(=O)[O-])CCCN2)n1', 'O=C(CNc1c(Br)cccc1[N+](=O)[O-])c1ccccc1', 'Clc1nnnn1C1CCN(c2cccc[nH+]2)CC1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn2cccc12', 'O=[N+]([O-])c1ccncc1[C@@H]1OCCN(c2ccc(OS(=O)(=O)c3ccccc3)nc2C2CC2)C1', 'OC[C@H](Cn1ccc(=O)[nH]c1=O)Nc1ccc2c(c1)OCO2', 'Clc1cc(Br)cn1N=C\\c1ccc(Br)cc1', 'Clc1ccccc1-c1csc(CN[C@@H]2CCC[C@@H](C(F)(F)F)C2)n1', 'Clc1cc(/C=Cc2[nH]nc3c2CCCC3)c(OCC(F)(F)F)cc1', 'Clc1cc(/N=C/c2n[nH]n3cccc23)ccc1O', 'C=C[C@@H](S(=O)(=O)c1ccc(C)cc1)c1nc(C)cc(C)c1[N+](=O)[O-]', 'Oc1ccc([N+](=O)[O-])c(CC[NH2+]C2CC2)c1Cl', 'Oc1cc(OCOCC2CCN(c3ccccc3)CC2)ccn1', 'Clc1ccc(-c2cccs2)nc1NCC[NH+]1CCCCC1', 'O=C(C1CCN(C(=O)CN2CCOCC2)CC1)Nc1ccc([N+](=O)[O-])cc1', 'Clc1cccc(-c2n[nH]cc2C2=NN=C(Sc3nccn3Cc3ccccc3)C2)c1', 'Clc1ccsc1S(=O)(=O)N1CCC(N(Cc2ccncc2)C(=O)c2ccc(F)cc2)CC1', 'Oc1cccc(S[C@@H]2CC(c3ccccc3)C(=O)N2c2ccc(C(C)=O)cc2)c1C(F)(F)F', 'CNS(=O)(=O)c1c(N=N/Nc2ccc(Cl)cc2C)nc2ccccc2n1', 'Clc1nnnn1-c1cnn(C[C@@H]2CCCO2)c1', 'Clc1ccccc1Nc1cncc(-c2ccc(F)cc2)n1', 'Clc1ccc(-c2nonc2Cc2ccc(Cl)cc2)cc1', 'Clc1ccc(-c2n[nH]c(Nc3ccc(Cl)cc3)n2)cc1', 'Clc1ccc(-c2cccnc2)nc1O[C@@H]1CCOC2(CCCCC2)C1', 'O[C@@H]([C@@H](O)c1ccccc1)c1ccc2c(c1)CCC2', 'Clc1ccccc1Oc1cnc(F)cc1', 'Clc1ccccc1-c1ncc[nH]c1=O', 'Clc1ccc(-c2n[nH]cc2Br)cc1', 'Clc1ccc(-c2n[nH]cc2Br)cc1', 'Clc1ccc(-c2n[nH]cc2Br)cc1', 'Clc1ccc(-c2n[nH]cc2-c2ccccc2F)cc1', 'Clc1ccc(-c2n[nH]cc2-c2ccccc2F)cc1', 'Clc1ccc(-c2ccccc2Cl)nc1O', 'Clc1cc(Cl)cnc1C1CCCCC1', 'O[C@@H]1[C@@H](c2ccccc2)Oc2ccccc21', 'Clc1ccccc1Cn1cc[nH+]cc1=S', 'Clc1nnnc(NCc2ccc(Cc3cc(F)ccc3F)s2)c1', 'O=[N+]([O-])c1cc(Br)c2ccccc2c1OCc1ccccc1F', 'Clc1ccc([C@@H]([NH2+][C@H]2CCCc3ccc(F)cc32)C2CC2)cc1', 'Nc1[nH]ncc1Nc1ccc(C(=O)Nc2ccccc2F)cc1Cl', 'Clc1ccc(-c2nonc2CNc2cc(Cl)ccc2Cl)cn1', 'Oc1cc(Cn2nc(Cl)ccc2=O)ccc1CO', 'Oc1cccc(O[C@H](C(=O)[O-])C[NH+]2CCN(Cc3ccccc3)CC2)c1', 'O=[N+]([O-])c1cccc2c1[nH]c(=S)n2CC1CCOCC1', 'Cc1cn[nH]c1C(=O)N[C@H]1CC[C@@](CO)(C(C)(C)C)C1', 'Clc1ccc(CCO/N=C(\\C)CCc2ccccc2)cc1', 'Clc1ccccc1Nc1cnc(Br)s1', 'COc1ncncc1CNCc1cc(F)ccc1F', 'CN(c1ccccc1)S(=O)(=O)c1ccc([N-]S(=O)(=O)c2cccc(F)c2)cc1', 'Clc1ccc(-c2ccccc2CN2CCC[C@](O)(C(F)(F)F)C2)cc1', 'CCCCn1ncc2c1N=C(C)O[C@@H]2c1ccccc1C', 'NS(=O)(=O)/C(=C\\NCc1ccccc1)n1cc(Br)c([N+](=O)[O-])c1', 'O=[N+]([O-])c1ccc(CCNC2=Cc3ccccc3CS2)cc1', 'Oc1ccc([C@H](N[C@H]2CCC[C@@H]2CO)N2CCOCC2)cc1', 'Clc1ccc([C@@H](CNC(=O)[C@@H]2CC(=O)N(SCc3ccco3)C2)c2cnn(-c3ccccc3)c2)o1', 'COC(=O)c1cnn(Sc2nncn2C2CC2)c1C', 'Clc1ccc([C@@H](NS(=O)(=O)c2ccc(F)cc2)NC(=O)C2CC2)cc1', 'O=C(CS(=O)(=O)c1c(F)cccc1F)NC[C@@H](O)C1CCCCCC1', 'Oc1cc(Br)c(Cl)cc1NCC[C@H]1CCCCCC[NH2+]1', 'O=C(CC12CC(CC(CO)C1)CCC2)Nc1ccc(F)c(Cl)c1', 'Clc1ccc(Br)cc1C1CC[NH+](C[C@@H](O)C2CCOCC2)CC1', 'O=C(c1c[nH]cn1)N1CC[C@@](CO)C1', 'Nc1[nH]nc2c1CCCCC[NH+](Cc1ccccn1)C2', 'Clc1ccsc1[C@@H](N[NH+]1CCC[C@@H](O)C1)c1ccccc1', 'Oc1cccc(Br)c1CNCc1ccc(Br)c(F)c1', 'NS(=O)(=O)C[C@@H](Cc1cn2ccccc2c1)c1ccccc1', 'O=C(/N=C/C1=C(N)NC(=O)N(c2ccc(Cl)cc2)c2ccccc21)c1ccccc1', 'O=C(c1ccsc1)c1ccc(COCCO)cc1', 'O=C(COCCC(=O)[O-])NCC1(c2ccc(F)cc2)CC1', 'O=C(C1CCN(c2ncccc2Cl)CC1)N1CC=C(c2ccccc2)CC1', 'Clc1ccccc1Oc1cc(Cl)c(N2CCOCC2)c(F)c1', 'OCc1ncc[nH]1', 'O=C(CSCNO[C@@H]1CCCOc2ccccc21)N[C@H]1CCc2ccccc21', 'Clc1cccc2ccc(NCc3cc(F)c(F)cc3F)cc12', 'NS(=O)(=O)C[C@@H](c1ccccc1)N1Cc2ccccc2C1', 'O=C(c1cnn2c1NCCCC2)N1CC(c2ncon2)C1', 'Clc1ccc(-c2n[nH]c(NC[C@H]3CCCC[NH2+]3)n2)cc1', 'Oc1ccc(F)cc1NSc1ccc2nnc(-c3ccccc3)n2c1', 'O=C(CS(=O)(=O)[C@H]2CCOc3ccccc32)N1CCc2ccccc21', 'Clc1cccc(CC[NH2+]C2CCN(CCO)CC2)c1', 'O=C(/N=c1\\oc2ccccc2[nH]1)c1cccc(C(F)(F)F)c1F', 'C[C@@H](NS(=O)(=O)c1ccccc1F)C(=O)NCc1ccc(F)cc1', 'Clc1ccc(Cc2nc3ccccc3[nH]2)nn1', 'Clc1ccc(-c2cc[nH]n2)nn1', 'Clc1ccc(-c2cc[nH]n2)nn1', 'O=[N+]([O-])c1cc(C(=O)O[C@H]2CCN(c3ncnc4sccc34)C2)cc([N+](=O)[O-])c1', 'O=C(CNC(=O)c1ccccc1)N/N=C/c1ccc2ccccc2c1', 'N#C[C@@H](Sc1cnccn1)Cc1cc(F)cc(F)c1', 'Clc1ccsc1CN1CC[C@H]2[C@H](C2)C1', 'O=C(c1cccs1)[C@H](CO)c1ccncc1', 'Cc1sn(-c2ccccc2)c(=O)c1S(=O)(=O)N1CCC[C@H]1Cc1ccccc1', 'Clc1cccc2nc(-c3ccco3)nc([O-])c12', 'Clc1cc(F)cc(Oc2ccncc2F)c1', 'Clc1ccc(C(=O)Nc2ccc(Oc3c(F)cccc3F)o2)cc1', 'O=[N+]([O-])c1ccc(N[C@H](c2ccccc2)C2CC2)cc1F', 'Clc1nnc([S-])n1CCOC(=O)c1ccc(Cl)c(F)c1', 'N=Cc1c([N+](=O)[O-])cccc1C(=O)NC[C@@H](O)C(=O)c1ccccc1', 'O=C(C[NH+]1CCC2(CC1)NCCN2C(=O)c1ccc2c(c1)OCC(=O)N2)OCCO', 'NS(=O)(=O)/C=C(\\CC#N)C(=O)Nc1ccc2c(c1)OCO2', 'ClC[C@@H]1C[C@@H](O)c2ccccc21', 'Clc1ccccc1C(CCCO)C[NH2+]C[C@@H]1CCCO1', 'Clc1ccc(Br)cc1C[C@@H]1CCCN(C(=O)c2cc([N+](=O)[O-])ccc2F)C1', 'Clc1ccc(-c2nnnn2/N=C/c2ccc(Cl)cc2Cl)cc1', 'Oc1cccc2ncn([C@@H]3CCCC[C@@H]3O)c12', 'Clc1ccccc1OCCCSc1nnc2n1CCCC2', 'Clc1ccccc1C1=NN=C[C@@H]1C(=O)NC1(c2ccccc2)CC1', 'Clc1cc(Cl)c(N2CCOC[C@H]2C2(CO)CC2)cc1Cl', 'Oc1cc(-n2cncn2)ccc1CNC1=Cc2ccccc2OC1', 'N=CC[C@@H]1CCCN1c1ncc(C(=O)[O-])cc1Cl', 'Clc1ccc(C(CCCO)C(F)F)cc1', 'Oc1cc(OCCn2cccc2Cl)c2ccccc2n1', 'COc1nnnc(N[C@H](C)c2ccc(F)c(F)c2)c1Cl', 'NS(=O)(=O)/C=C/CC(=O)Nc1cnn(C)c1', 'O=C(/N=C/c1ccc(F)cc1)c1ccc(NC(=O)c2cccc([N+](=O)[O-])c2)cc1', 'Clc1ccccc1Oc1cc(NCc2ccccc2Cl)ncn1', 'O=C(CCc1nc(-c2ccccc2)cs1)Nc1ccc(F)cc1Cl', 'OC[C@@H](Cc1cc(Cl)cc(Cl)c1)Nc1cc(OCc2ccccc2)ncn1', 'O=C(c1ccsc1)[C@@H]1[C@@H]2C[C@@H]2CC[C@@H]1C[C@@H]1CCCO1', 'COC(=O)[C@@H](C)NS(=O)(=O)c1ccccc1Cl', 'Clc1ccccc1[C@H](NNC(=O)C1CCC1)c1ccccc1', 'Clc1cc(Cl)nc(N2CCC(CSc3ccc(O)cc3)CC2)c1', 'N#C[C@@]1(C2CC2)Nc2cccnc2N1', 'Clc1ccccc1NCC(=O)N1CCN(S(=O)(=O)c2ccc(F)cc2)CC1', 'Oc1cc(F)nc(NC[C@H]2CCCO2)c1', 'O=[N+]([O-])c1c(-c2ccsc2)ncnc1NCc1ccccn1', 'O=C(/C=C/c1cccc(Cl)c1)NCc1cc(Br)ccc1Br', 'ClCc1csc(OC[C@H]2CCCO2)n1', 'Clc1cc(C(=O)NCCCc2ccsc2)cs1', 'Oc1ccc(F)cc1-c1ccccc1[C@@H]1CCCN(C(=O)c2cnnc3c2s3)C1', 'Clc1ccsc1S(=O)(=O)c1ccc(OC2CCCC2)cc1', 'Oc1cc(Br)cc(CCc2nnc(-c3ccc(Cl)cc3)o2)c1', 'Clc1cccc([S@@]Cc2cccc3c2OCO3)c1', 'O=[N+]([O-])c1cc(C(=O)N2CCCc3ccccc32)ccc1N1CCCC1', 'N#C/C(=C\\c1cn(CC(=O)[O-])c(C(=O)N2CCOCC2)c1C#N)c1ccco1', 'O=[N+]([O-])c1ccc[nH+]c1N1CCC(n2cncn2)CC1', 'OC[C@@H](NCc1cc(Br)cs1)c1ccc(Cl)cc1', 'Clc1cc(Br)cn1C[C@@H]1CCCCCC1', 'Oc1ccc(/C=C(/Cl)c2ccccc2[N+](=O)[O-])cc1', 'N=CCS(=O)(=O)Cc1nc2nc(C)cc([O-])n2n1', 'COc1nn(-c2ccccc2)c(C)c1C(=O)NC[C@@H](c1cccs1)[NH+](C)C', 'Clc1ccccc1Oc1cnc2nc(-c3ccc(Br)o3)[nH]c2c1', 'Clc1nnnn1-c1c(-c2ccccc2)c[nH]c1CSc1nc2ccccc2n1CC(F)F', 'O=C(c1ccsc1C[NH+]1CCCCC1)c1ccc(Br)cc1', 'COc1ncncc1C(=O)Nc1cc(S(=O)(=O)C(=O)N2CCCC2)ccc1F', 'O=[N+]([O-])c1c(-c2cc(I)cc(Cl)c2)ncnc1N1CCc2ccccc21', 'Oc1cc(F)c2nc([C@H]3CCCN3C(=O)Cc3ccco3)oc2c1', 'Clc1ccc(Br)cc1-n1cccn1', 'Clc1ccc(Br)cc1-n1cccn1', 'Clc1ccc(Br)cc1-n1cccn1', 'Clc1ccc(Br)c(-n2cccn2)c1', 'Clc1cc(CC[NH+]2CCN([C@@H]3CCOC3)CC2)ccc1Cl', 'Clc1ccc(Br)cc1S[C@H]1Cc2ccccc2C1', 'CN(c1ccccc1)S(=O)(=O)Cc1ccccc1N', 'Clc1cccc(C2(CO)CC2)c1Cl', 'Cc1ncc([C@H](C)[NH2+]C(C)(C)C)s1', 'O[C@@H]1c2cc(F)cc(Cl)c2C(=O)N1CCCCn1cccn1', 'ClCc1noc(-c2ccccc2)n1', 'O=C(CS(=O)(=O)[N-]C[C@@H](O)c1ccccc1)Nc1ccc(F)cc1', 'O=C(/N=C/CCN1C(=O)c2ccccc2C1=O)NCc1cccc(Cl)c1', 'Clc1ccc(C(CCBr)COCCO)s1', 'Clc1cc(Br)cnc1-c1ccc(Cl)cc1', 'Clc1ccc(Br)cc1Sc1ccc(Br)cc1', 'O[C@@H](S[C@@H](c1ccccc1)c1ccncc1)C1CC1', 'Oc1cccc2ncn([C@@H](Cc3ccccc3)c3ccccc3)c12', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1Sc1ccc(Cl)cc1Cl', 'Clc1ccccc1OCc1cc(COC(F)(F)F)cs1', 'Clc1cccc2c(OCCOc3ccccc3Cl)ncnc12', 'Clc1cc(/C=N/N(c2ccccc2Cl)C2CC2)c[nH]1', 'N[C@@H](Sc1nc(-c2ccccc2F)n[nH]1)C1CC1', 'COc1ncncc1CSCOC(=O)Cc1ccccc1', 'Nc1nc(CNCc2ccc[nH]2)nc2ccc(F)cc12', 'Oc1ccc(F)cc1Cc1c[nH]c(Nc2ccccc2Cl)n1', 'Oc1ccc(/C=C2\\SC(=S)N(CC(=O)NCc3ccc(F)cc3)N2Cc2ccccc2)cc1', 'Oc1ccc(F)cc1S(=O)(=O)N(C1CC1)[C@@H]1CCS(=O)(=O)C1', 'ClCc1csc(NCc2cccc(OCC3CCOCC3)c2)n1', 'Clc1ccccc1[C@H]1C(c2ccccc2)=Nc2ncnn21', 'O=C(/N=c1/o/[n+](=N/C(=O)OCc2ccccc2)nc1-c1cccs1)c1ccc(Cl)cc1', 'Oc1cc(F)ncc1Sc1ncnc2ncnc(Nc3ccccc3)c12', 'OC1([C@H]([O-])c2ccc(-n3cccn3)cc2)CCOCC1', 'COc1nc2c(c(=O)[nH]1)[C@@H](c1c(F)cccc1Cl)[C@@H](C(=O)Nc1ccccc1Br)CS2', 'O=C(c1ccc[nH]c1=O)N(Cc1nc2ccccc2n1Cc1ccccc1)S(=O)(=O)c1ccccc1', 'Clc1ccc([C@@H](CO)[NH2+]C2CC2)s1', 'O=[N+]([O-])c1ccn(C[C@H](O)c2cccs2)c1', 'Nc1cnn([C@@H]2CCC(=O)N(C3CCCCC3)C2)c(=O)n1', 'Clc1nnc(CNc2nccc(OCc3cccs3)n2)o1', 'Oc1cccc2nc(NC[C@@H](O)Cc3ccccn3)cnc12', 'O=C(C1CC1)N1CC(c2nc(C(F)(F)F)co2)C1', 'Nc1cnnn1Cc1cc(Br)ccc1O[C@H]1CCOC1=O', 'Clc1ccc([C@@H](Cl)C2CC2)c2ncccc12', 'Oc1cccc(N2C(=O)[C@@H](Cc3ccccc3)N2c2ccc(C(=O)[O-])cc2)c1', 'Clc1cc(/C=C(/Cl)[C@H]2COc3ccccc32)cc([N+](=O)[O-])c1', 'Clc1ccccc1N1N(CCSc2ccc([N+](=O)[O-])cc2)CC1', 'O=[N+]([O-])c1cc(Cl)c(S(=O)(=O)N/N=C/c2ccc(Br)cc2F)cc1Cl', 'COc1ncc(NC(=O)N2C[C@H](C)O[C@H](C)C2)c(C)c1OC', 'O=C(/N=c1c(N)nc2c(Cl)cccc12)c1ccc[nH+]c1', 'Oc1ccc(O[C@H]2C=C[C@H](c3ccccc3)C2)cc1Cl', 'Clc1ccccc1Oc1cc(S(=O)(=O)N2CCN(CC(F)(F)F)CC2)ccc1Cl', 'Clc1ccc(C[NH2+][C@@H]2CCC[C@H](c3nc(C(F)(F)F)cs3)C2)cc1', 'O=C(c1ccsc1)c1c[nH]c2ccccc12', 'OCS[C@@H](Cc1ccccc1Cl)c1c(F)cnc(F)c1F', 'Clc1ccccc1OCc1csc(-c2ccccc2)n1', 'O=C(CSCCNC(=O)NCc1ccc(F)cc1)Nc1ccc([N+](=O)[O-])cc1', 'Clc1nnnn1c1nnnn1C1CCCC1', 'Clc1cccc([C@@H](Nc2nccc(C(N)=O)c2)C2CC2)c1', 'Clc1ccsc1[C@H](N[C@@H]1CCSC1)c1ccccc1', 'Clc1nncc(F)c1-c1ccc(Cn2ccccc2=O)cc1', 'Clc1ccccc1OC1(OC2CCCC2)CCOCC1', 'Oc1ccccc1-c1c([C@@H](Cl)c2ccccc2)ncnc1Cl', 'Clc1ccsc1CN(Cc1cccc(F)c1)c1ccccc1', 'Clc1ccccc1-c1nc(CN2CCOCC2)nc2nc[nH]c12', 'ClC(c1cc2cccc(Cl)c2o1)[C@H]1CCC[NH+]2CCC[C@@H]12', 'Clc1cc(F)ccc1[C@@H](O)Cc1ccc[nH]1', 'Clc1ccccc1CNc1nnc(Cn2ncnc2-c2cccnc2)s1', 'NS(=O)(=O)N[C@H]1C=Cc2ccc(C(=O)N3CCCC3)cc2N1', 'N[C@@H](c1cccs1)CO[C@@H]1C[C@H]1C', 'O=C([O-])COc1nc(CCc2ccsc2)n[nH]1', 'Clc1ccc(-c2nnnn2Cc2cccs2)cc1', 'OCc1n[nH]c(NCc2ccccc2Cl)n1', 'O=C(c1cccs1)C(=O)NCc1ncc(Cc2ccccc2)o1', 'O=C(/C=C1/S/C(=C/c2cccc(F)c2)N1c1cccnc1)N1CCCC[C@@H]1CNc1ccc2c(c1)CCCO2', 'Clc1ccc(-c2ccccc2N2CCC[NH2+]C[C@H]2CS(=O)(=O)[O-])cc1', 'Clc1ccccc1Nc1nc(Cl)nc(NN2CCOCC2)c(=O)c1C#N', 'O=[N+]([O-])c1c(F)cc(C[S@@](=O)/C=C(\\Cl)Cc2ccccc2)c(Cl)c1O', 'O=[N+]([O-])c1c(OCc2ccc(F)cc2)csc1NCc1ccc(F)cc1', 'O=C([C@H](c1ccc(Cl)cc1Cl)N1CC[NH+](C2CCCCC2)C1)N1CCOCC1', 'Oc1ccccc1/N=C1/C(=C/c2ccc3c(c2)OCO3)N=C2CCC[C@@H]21', 'O[C@@H](CSc1n[nH]c2c1CCCC2)c1ccccc1O', 'O=C(Cc1noc(C2CC2)n1)[C@@H]1CCCCN1S(=O)(=O)c1ccccc1', 'O=C(/C(c1cccs1)c1ccc(Br)cc1)c1ncc[nH]1', 'O=C(CS(=O)(=O)[C@H]1CCCc2ccccc21)N/N=C/c1cnc2ccccc2c1', 'Clc1nncn1/n1/nc(CSc2ccc(F)cc2)n2ccnc21', 'Clc1ccccc1NCc1cncn1Cc1cccs1', 'Oc1cccc(NC2=NC(=O)C3=Cc4ccccc4C(c4ccccc4)N32)c1', 'O=C(c1c[nH]cn1)[C@H](Nc1ccsc1)c1ccccc1', 'Clc1cc(Br)c(COC2CC2)cn1', 'Nc1c(C(=O)N2CC[C@@H](c3ccco3)C2)cnn1C(C)C', 'C[C@@H](c1ncnn1C)C(=O)[O-]', 'O=C(/N=C\\Nc1cccc(S(=O)(=O)N2CCOCC2)c1)C1CCC(c2ccccc2)CC1', 'N=Cc1c(F)cccc1OCCOC(F)F', 'Clc1ccc(Br)cc1N(CN1CCOCC1)C1CC1', 'ClCc1nnc([C@@H]2COc3ccc(Cl)cc3C2)n1CC1CC1', 'Clc1ccc(Br)cc1/C=N\\OCCN1CCCC1=O', 'Oc1cccc(S[C@@H](c2ccccc2F)N2CCC(c3c(F)cccc3F)CC2)c1', 'Clc1ccc(-c2cc(Cl)ccc2N2CCCCC2)cn1', 'Clc1cccc(F)c1[C@@H](O)c1cccc(OCc2ccccc2)c1', 'O=C(c1cn2ccsc2n1)C1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1', 'Clc1ccc(-c2ccccc2N2CCN(c3ccc(Cl)c(Cl)c3)CC2)cc1', 'Clc1cccc(-c2cccn2CC[NH2+]C2CC2)c1', 'Oc1cc(COCc2ccncc2)ccc1O', 'CNS(=O)(=O)CC(=O)N1CCN(C(=O)c2cccc(C)c2)CC1', 'Clc1cccc(-c2nnc([C@H]3CCCN(S(=O)(=O)CC4CCCCCC4)C3)o2)c1', 'Clc1nnnc(-c2ccco2)n1', 'Clc1nnnc(-c2ccccc2)n1', 'Clc1nnn(-c2ccccc2)n1', 'Clc1ccccc1-c1nc(-c2ccncc2)no1', 'Clc1ccc(-c2nonc2)nn1', 'Clc1ccc(-c2nonc2)nn1', 'Clc1ccc(-c2nonc2)nc1', 'Clc1ccc(-c2nonc2)nc1', 'Clc1ccc(-c2nonc2)cn1', 'Clc1ccc(-c2nonc2)cn1', 'O=C(CCOn1cccn1)Nc1ccc2c(c1)OCO2', 'O=[N+]([O-])c1cc(SCCC(=O)N2CCC[C@@H](c3nc4ccccc4s3)C2)ncn1', 'O[C@@H](c1cc(Br)cnc1Cl)Cc1ccc(Cl)cc1F', 'Oc1ccc(/C=N/c2ccccc2Cl)cc1', 'O=C(CNCc1cccnc1)Nc1cc(Cl)ccc1[N+](=O)[O-]', 'Clc1cccc2cc(OC[C@@H]3CCCO3)oc12', 'O[C@@H]1C[C@@]1(O)CCc1ccc(Cl)cc1', 'CNC(=O)[C@@H]1C[C@@H](O)C[NH+]1CC(=O)Nc1ccccc1F', 'Oc1cccc2ncn(/N=C/c3ccccc3)c(=O)c12', 'Clc1cccc([C@@H]([NH2+]Cc2nccs2)C2CC2)c1', 'Clc1ccc(C(N)=O)c([N+](=O)[O-])c1Sc1ccccc1', 'O=C([O-])/C(CSc1ccccc1)C(=O)Nc1ccccc1', 'O=C(CNS(=S)(=O)c1ccccc1)Nc1ccc(F)c(Cl)c1', 'Oc1ccccc1CC[NH2+]Cc1ccc([N+](=O)[O-])cn1', 'Clc1cc(Cl)cnc1SCC1CCOCC1', 'O=C(CS(=O)(=O)C[C@H]1Sc2ccccc2NC1=O)NC[C@@H](O)c1ccccc1', 'NS(=O)(=O)/N=C(\\[O-])CCS(=O)(=O)c1ccccc1N(C)C', 'ClC[C@H]1CC[C@H](Cc2ccc(F)cc2)[N+]12CCCCCC2', 'CNS(=O)(=O)c1c(F)cc(C(=O)NC2CCN(C(=O)COc3ccccc3)CC2)cc1C', 'Clc1cc(/C=N\\N2CCC[C@@H](C(=O)Nc3ccc([N+](=O)[O-])cc3)C2)ncn1', 'O=C([O-])[C@@H](Cc1ccc(F)cc1)N1CCCCC1', 'Clc1ccccc1Oc1cnccn1', 'Clc1ccccc1Oc1cnccn1', 'Clc1ccccc1Oc1cnccn1', 'Clc1ccccc1Nc1ncccn1', 'Oc1ccc(F)cc1-c1nnc(SCCc2ccccc2)n1-c1ccccc1', 'Clc1ccc(COc2nc[nH]c2Br)cc1', 'Clc1ccc(COc2nc(Br)ccn2)cc1', 'Clc1ccc(C2(Cl)CCC2)cc1', 'Cc1cc(F)ccc1CCN(C(=O)[C@H](C)[N+](C)(C)C)c1ccccc1', 'Clc1ccc(CO)c(Oc2ccccc2Br)n1', 'Oc1ccc([N+](=O)c2cc(C(=O)NCCc3c[nH]c4ccccc34)ccc2F)cn1', 'Oc1cc(C=NNCc2ccc(CN3CCOCC3)o2)c(F)cc1Cl', 'O=[N+]([O-])c1cc(Br)ccc1OCCN1CCC[NH2+]CC1', 'O=[N+]([O-])c1cc(Br)c(O)c([N+](=O)[O-])c1OCCC[NH+]1CCCC1', 'O=C(COc1cc(Cl)ccc1F)NNC(=O)CCC(=O)c1ccc(F)cc1F', 'Clc1ccc(-c2cc(Br)ccc2ONCCCO)c(C(=O)[O-])c1', 'O=C(CN(c1cncc(C(F)(F)F)n1)S(=O)(=O)c1ccccc1)N1CCCCCC1', 'Clc1nnnn1C1CCN(C(=O)c2cc(Cl)c3c(c2)OCCO3)CC1', 'Clc1cc(/N=C/c2ccccc2)c(Br)o1', 'Oc1cc(F)ncc1Cn1cnc2ccccc21', 'NS(=O)(=O)/N=C/CCc1ccc2c(c1)OCO2', 'Clc1ccccc1CCC(=O)Nc1ccc2c(c1)CCC2', 'O=C(CCS)/C(=N\\c1ccccc1)Sc1nncn1CC(=O)[C@H](O)c1cccc(Br)c1', 'Clc1ccc(COc2ncccc2F)cc1', 'O=[N+]([O-])c1ccncc1N[C@@H]1c2ccccc2CC[C@@H]1O', 'Nc1cn[nH]c1-c1ccc(NC(=O)c2ccccc2O)cc1', 'Cc1nccn1-c1ccc(NC(=O)N[C@H](C)c2c(C)nn(C)c2C)cc1O', 'OC[C@H]1O[C@H](O)CS[C@H](OCc2ccccc2)C1', 'Clc1cccc([C@@H](N[C@@H]2CCc3nc[nH]c3C[NH+]2CC(F)(F)F)c2cccnc2)c1', 'Clc1ccc(C(F)(F)F)c(NCc2ccc(F)cc2Br)c1', 'Clc1ccsc1-c1cnnn1CC[C@@H]1CCCC[NH2+]1', 'Clc1cc(F)cc(N2CCCC[C@@H]2Cc2ccccc2)c1', 'ClCc1csc(C(=O)N2CCN(Cc3ccccc3Cl)CC2)c1', 'Clc1cc(/C=N/n2cc(S(=O)(=O)Cc3ccccc3)ccc2=O)ccc1C1CC1', 'Nc1cn[nH]c1-c1ccccc1Br', 'NS(=O)(=O)CCN(C(=O)C1CC1)C1CCC(c2ccccc2)CC1', 'O=[N+]([O-])c1cc(S(=O)(=O)Nc2ccc(Cl)c(Cl)c2)cc(Cl)c1Cl', 'O=[N+]([O-])c1c(-c2ccc(Br)cc2)n[nH]c1N[C@H]1CCC[C@@H](C(F)(F)F)C1', 'Clc1cccc(C[NH2+][C@@H]2CCCC[C@@H]2c2cccc(F)c2)c1', 'O=C([C@H](CCSc1ccccc1)c1ccsc1)N1CCCCC1', 'NS(=O)(=O)N[C@@H](c1ccccc1)c1ccc(Cl)cc1', 'Oc1cccc2nnc3cc([C@@H](CCO)c4ccccn4)nc([O-])c3c12', 'Cc1nnc(SCCN2C(=O)N[C@]3(CC[C@@H](O)CCC3)C2=O)s1', 'Oc1cc(O[C@@H]2O[C@@H]3CC[C@@H]2C3)ccc1Oc1ccccc1', 'Clc1ccc(Br)cc1-c1cccnc1', 'NS(=O)(=O)C[C@@H]1[NH2+]CC[NH+](Cc2ccccc2)C1', 'Clc1ccc(Br)cc1Cc1ccnc(F)c1', 'Clc1ccsc1/N=N/CCOc1ccccc1Cl', 'O=C(c1c[nH]c(Nc2cc(Cl)ccc2F)c1)c1ccccc1', 'Clc1ccc([C@@H]([C@@H]O)N[C@@H]2CCc3ccccc32)cc1Cl', 'Clc1ccc(-c2cccnc2)cc1CN1CCN(c2ccccc2N)CC1', 'NS(=O)(=O)Cc1ncco1', 'Clc1ccccc1N1N=C(c2cccnc2Cl)C[NH+]1Cc1ccc(F)cc1', 'C=CCn1cc(NC(=O)c2[nH]nc(C3CC3)c2C)c(C)n1', 'Oc1cc(-n2cnc3ccccc32)cc([N+](=O)[O-])c1', 'Clc1cc(Br)ccc1S[C@@H]1CCOC([O-])C1', 'O=C(CS(=O)(=O)CC[C@H](O)c1ccccc1)N(Cc1ccc(F)cc1)C1CC1', 'O=C(C1CCN(Cc2cccc(Cl)c2)CC1)N1CCC2(CC1)OCCO2', 'O=C(Cc1nonc1C(F)(F)F)Nc1ccc(Cl)cc1', 'O=[N+]([O-])c1c(-c2cc(=O)[nH]c3ccccc2c23CCOCC2)nccc1I', 'Clc1ccc(Br)cc1-n1nnc2ccc([N+](=O)[O-])cc21', 'Clc1ccc(Br)c(-n2nnc3cc([N+](=O)[O-])ccc32)c1', 'CN=N[C@]1(C(=O)OC)CCc2ccccc21', 'O=C(c1ccsc1)Nc1ccn(CCOc2ccccc2F)n1', 'Clc1ccc(Br)cc1/N=C\\NC1CCN(c2ccc(F)c(F)c2)CC1', 'Clc1cc(N2CC[C@H](Cc3ccccc3)C2)nc2ccccc12', 'Clc1ccccc1O/C=C1/C(=N/CC2CCCCC2)N=N1', 'Oc1ccc(Oc2cn(Cc3ccccc3)nn2)cc1Cl', 'Oc1ccc(/N=c2/[nH]ccnc2COc2ccccc2)cc1Cl', 'C[C@@H]([C@@H](O)CCO)N(C)C(=O)c1ccc(Br)cc1F', 'O=C(c1cn2ccsc2n1)c1ccc(S(=O)(=O)N2CCCCCC2)cc1', 'N[C@@H]1c2ccccc2CCC1=O', 'Clc1cccc(-c2n[n-]nc2CN2CCCC[C@H]2c2ccccc2)c1', 'Clc1cccc2[nH]ncc1n1c(C(=O)OCC(F)(F)F)c1C[NH+](Cc1ccccc1)C2', 'Oc1ccc([N+](=O)c2onc(C3CC3)n2)cc1OC[C@@H]1CCCO1', 'OCc1n[nH]c(Nc2ccc(F)cc2)c1-c1ccccc1', 'Clc1ccccc1O/C=C[C@@H]1CN2CCCC[C@H]2CO1', 'Clc1cc(/C=N/c2cccnc2)ccc1Cl', 'Clc1ccc(Br)cc1[C@@H]1O[C@H](CO)[C@H](O)[C@H]1Cc1ccccc1', 'Oc1cc(C[NH2+][C@@H]2CCCc3ccccc32)c(F)cc1Br', 'O=C(c1cccc(F)c1F)N1CCC(OCc2cccc(F)c2)CC1', 'Cc1nc(C(=O)N2CCC[C@@H]2C(=O)c2c(C)cc(C)cc2[N+](=O)[O-])c2ccccc2n1', 'Clc1ccc(Br)cc1-c1ccn(C[C@H]2CCCO2)n1', 'Oc1cc(-n2cc(C(F)F)ccc2=O)ccc1C(=O)Nc1cc(F)ccc1[N+](=O)[O-]', 'O=C([C@H]1c2cc(C(F)(F)F)ccc2NC1=O)N1CSC[C@H]1[C@H]1CCCO1', 'O=[N+]([O-])c1c(Cl)nsc1C(=O)N1CC[C@@H](Oc2ccc(F)cc2)C1', 'Oc1cc(-n2nncc2NCCc2ccc(CCl)cc2)ccc1O', 'Clc1ccccc1O/N=C/Sc1ccccc1[N+](=O)[O-]', 'O=C(CNS(=O)(=O)/C=C/c1cccc(Br)c1)N1CCc2ccccc21', 'Clc1nncc(CSc2nccnc2C2CC2)n1', 'COCc1c(-c2ccccc2)[nH]c(=S)n1C', 'Clc1ccc(CCOc2ccc3ccccc3c2)cc1', 'Clc1ccccc1[C@H](C#N)C(=O)Cc1cccnc1', 'Clc1ccc(-c2csc3[nH]c4ccccc4c(=O)c23)cn1', 'Clc1ccsc1[C@@H]([NH3+])c1ccccc1', 'Clc1ccccc1-c1cscc1C[NH3+]', 'O=[N+]([O-])c1c(F)cc(OC[C@H](O)COc2cccc(F)c2)cc1Cl', 'O=C(CC(=O)Nc1ccc(Cl)cc1F)Nc1cccc(N2CCCC2)c1', 'Oc1cccc(OCc2nc(C3CCCCCC3)[nH]n2)c1', 'OCc1nc2ccc(F)cc2n1CCN1CCOCC1', 'Clc1cc(CNC[C@H](N)c2ccccc2)[nH]n1', 'Clc1ccc(-c2cccs2)nc1N1CCc2ccccc21', 'Clc1cccc(S[C@H]2C[NH2+]CCN2C(=O)c2ccccc2)c1', 'Clc1ccccc1-c1csc([C@@H]2CCCN(S(=O)(=O)CCN3CCOCC3)C2)[nH+]1', 'O=C([C@H]1c2ccccc2CCc2sccc21)N1CCOCC1', 'O=C(c1ccsc1)N(CCO)Cc1cccnc1', 'O=C(/N=c1/[nH]ccn1CC1CC1)c1csc(-c2c[nH]cn2)n1', 'Clc1ccccc1Nc1ccccc1Cl', 'Clc1nnn(-c2ccc(F)cc2)c1I', 'O=[N+]([O-])c1cccc(-c2nnc(CC[NH+]3CCCC3)o2)c1', 'Clc1ccsc1C[N-]/N=C/C[NH+]1CCCCCC1', 'ClC(c1ccc2c(c1)CCC[C@H]2[NH2+]C[C@@H]1CCCO1)c1ccc(F)cc1', 'Cc1nonc1CN(Cc1ccccc1)c1ncccn1', 'Clc1ccc(-c2cc(Cl)ccc2F)cn1', 'Clc1ccc(-c2cc(Cl)c(Cl)cn2)cc1', 'Clc1ccccc1-c1cnc2n1CCC[C@@H]2N[C@@H]1CCS(=O)(=O)CC1', 'Clc1ccccc1-c1cscc1Cn1ccnc1C1CC1', 'O=C(COc1ccc(I)cc1)Nc1ccnc(C(=O)[O-])c1', 'Clc1ccccc1O/N=[N+]/C=CSc1ccc2c(c1)OCO2', 'Oc1ccccc1Cc1ccccc1Br', 'Oc1ccccc1-c1ccccc1F', 'Oc1cccc(Br)c1Cc1ccccc1', 'Clc1cccc(-c2nnc(SCCC[NH+]3CCCCCC3)nc2-c2ccccc2)c1', 'O=[N+]([O-])c1c([O-])ncnc1[C@@H]1CCOC1', 'O=[N+]([O-])c1ccnc2ccc(NCCNS(=O)(=O)Cc3ccccc3)cc12', 'Clc1cc(Br)c([N+](=O)[O-])cc1N1CCC[C@H](CO)C1', 'O=[N+]([O-])c1ccc(NC[C@@H]2C[C@@H](O)C[NH+]2Cc2ccccc2)cc1F', 'O=C(/C=C/c1cccc(Cl)c1)NOCC1(O)CCOCC1', 'N=CC[C@@H]1C(=O)N[C@]2(CCc3ccccc32)C1(Cl)Cl', 'O[C@@H](Sc1nnc(C2CC2)n1C1CC1)C(=O)Nc1ccccc1', 'O=C(c1ccc(O[C@H]2CCOC2)cc1)N1CCCC1', 'O[C@@H](c1ccccc1Cl)[C@H]1CSCCSC1', 'Clc1cc(Cl)ccc1OS(=O)(=O)c1ccc2c(c1)CCCC2', 'Clc1ccc(Br)cc1NCC(=O)N1CCCC1', 'Oc1ccccc1Oc1c(Cl)ccc2c1NCCN2C(=O)c1ccccc1', 'Clc1ccccc1C1([O-])CC1', 'Clc1cc(/C=C/N=C(\\Nc2ccccc2)c2ccccc2)ccc1Cl', 'C[C@@H](Nc1cccc(Cl)c1)[C@@H]1CCC[NH2+]C1', 'Clc1ccccc1NCc1ccc(Cl)o1', 'Clc1cccc(Br)c1OCc1ccco1', 'Clc1cccc(F)c1Oc1ccccc1', 'Clc1cc(Br)cn1[C@@H]1CCO[C@]2(CCOC2)C1', 'Clc1ccccc1Oc1cc(-c2cccc(F)c2)ccc1O', 'O=C([C@H](SC1=NC=NC1=O)c1ccccc1)c1ccc(Br)cc1', 'O[C@@H]1c2ccccc2CC[C@@H]1O', 'N=Cc1c(-c2ccccc2)nnnc1Sc1ncnc(N2CCC(C)CC2)c1CC', 'Clc1cc(F)c(Cl)nc1NC[C@@H]1CCCN1C(F)(F)F', 'Clc1cc(F)cc2nccn12', 'Oc1cccc(S[C@@H]2CCC[NH2+]2)c1F', 'COc1ncc(CNS(=O)(=O)Cc2ccc(F)c(F)c2)cc1OC', 'Clc1ccccc1N1N(CC(=O)Nc2cccc(Cl)c2)CC1', 'Oc1cccc([C@H]2O[C@@H](c3ccccc3)S/C2=N\\C2=C/C=C\\c3nc4ccccc4n3-2)c1', 'Clc1cc(Br)c(CN2CCCC[C@H]2c2cnco2)cc1Cl', 'O=[N+]([O-])c1ccnc2ccccc12', 'O=[N+]([O-])c1ccnc2ccccc12', 'O=[N+]([O-])c1ccnc2ccccc12', 'O=[N+]([O-])c1ccnc2ccccc12', 'O=[N+]([O-])c1ccnc2ccccc12', 'ClCc1cncc(-c2ccc(O)c3ccccc23)n1', 'O=C(CCOc1ccccc1Cl)[C@@H]1N=NN=C1c1ccc2c(c1)OCO2', 'Clc1cccc(F)c1OCC(=O)NCc1ccc(C(=O)[O-])cc1', 'Clc1ccc(-c2cc(/[NH+]=C/c3ccco3)nc([S@](=O)c3ccccc3)n2)cc1', 'Clc1nncn1Cc1cnn(-c2ccccc2)c1', 'Clc1ccccc1Nc1nccn1/C=O', 'Clc1ccccc1-c1nc(Cc2ccncc2)no1', 'Clc1ccc(Cc2noc(-c3ccc(O)cc3)n2)cc1', 'Clc1ccc(Br)c(S(=O)(=O)N[C@@H]2CCCc3ccccc32)c1', 'CN[C@H](CCc1[nH+]ccs1)c1ccsc1', 'Clc1cc(/C=N\\O[C@@H]2CCC[C@@H]2[C@@H]2CCCO2)ccc1O', 'O=[N+]([O-])c1c(Cl)n2nccc2n1Cc1ccccc1Br', 'Clc1ccc(-c2nonc2Cn2cccc(Br)c2=O)cc1', 'ClCc1noc(-c2cnc(-c3ccc(Cl)cc3)nc2)n1', 'O=C([C@H](Sc1n[nH]c2ccccc12)c1cc2c(cc1F)OCCO2)N1CCSCC1', 'O=[N+]([O-])c1ccn(C[C@@H](Cl)Oc2cccc(F)c2)c1', 'Clc1ccc(-c2cc(/C=C/c3cccnc3)n[nH]2)cc1', 'Oc1ccc(F)c(Cl)c1COc1cnnc2ccccc12', 'Clc1cccc2nn(SC[C@@H]3CCCO3)nc12', 'Oc1ccc(F)c(Cl)c1N[C@@H]1CCS/C1=N/c1cccc(Br)c1', 'Clc1ccccc1Oc1cc(Nc2ncno2)ccc1F', 'Clc1ccccc1O/Cn1cnc(-c2ccc(Cl)c(F)c2)n1', 'Clc1ccc(Br)cc1-n1nccc1CSc1nnnn1C1CC1', 'Clc1cccc2ncncc12', 'Clc1cccc2ncncc12', 'Clc1cccc2ncncc12', 'Clc1cccc2cc(Oc3nc(-c4ncn[nH]4)cs3)ncc12', 'O=C(CS(=O)(=O)Cc1nnc(-c2ccoc2)o1)Nc1ccccc1-c1ccccc1', 'Clc1cccc2c1CC[C@@H]2c1ccc(Cl)cc1', 'COC[C@H]1[NH2+][C@@H](c2ccccc2OCc2cccc(C)c2)CC1', 'Clc1ccsc1[C@@H]1[NH+]=c2ccccc2=[NH+][C@H]1COC(F)(F)F', 'Nc1nc(NCC2CCC2)c2c(C(N)=O)cccc2n1', 'Clc1cccc(/N=N\\NC(=S)NNC(=O)Cc2ccccc2)c1', 'Clc1ccc(Br)c(S(=O)(=O)N2CCC(OCc3ccccc3)CC2)c1', 'CNC(=O)c1ccc([C@H](C)[NH2+]C2CCN(c3ccccc3F)CC2)cc1', 'Oc1ccccc1-n1nnnc1SCC(=O)N1CCC(C2c3ccccc3C2)CC1', 'Clc1cccc(S(=O)(=[NH2+])CC2C(C)(C)CS2)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])c(Br)sc1N[C@@H]1C=CS(=O)(=O)C1', 'Clc1ccccc1NS(=O)(=O)NC[C@]1(O)C(=O)NC([O-])C1', 'O=C(CC[C@@H]1[NH+]=c2ccccc2=[NH+]C1)N(Cc1cccs1)SC(F)(F)F', 'Clc1ccccc1CS(=O)(=O)CC1CC[NH2+]CC1', 'O=C(c1c[nH]nc1-c1ccccc1)N1CCN(c2cnccn2)CC1', 'ClCc1csc(OC2CC2)n1', 'O=C(c1ccsc1)Nc1ccccc1OC1CCCC1', 'Clc1ccsc1[C@H]1CCC[NH+](Cc2ccco2)C1', 'O=C(CS(=O)(=O)Cc1cccc(Br)c1)N1CCN(c2ccccc2)CC1', 'O=C(/N=C/N=C\\c1cccc(O)c1)[C@H]1CCCN(S(=O)(=O)Cc2ccccc2)C1', 'Nc1nonc1Cn1ncc2[nH]c3ccccc3c21', 'O=C(CNS(=O)(=O)c1cnc2c(c1)c(=O)n2Cc1ccccc1)NC1CCCCC1', 'Clc1ccc(COCc2cccc(C(Br)(F)F)c2)cc1', 'Clc1ccccc1NS(=O)(=O)N[C@H]1CCS(=O)(=O)C1', 'Clc1ccccc1NS(=O)(=O)NC1CCN(O)CC1', 'Oc1cccc2cc(CN3CCC[C@@H]3C(=O)c3cccc(C(F)(F)F)c3)oc12', 'O=C(CCc1ccsc1)Nc1ccc2nc(C3CC3)oc2c1', 'O=C(c1ccsc1)Nc1ccc(C(=O)Nc2ccccc2)cc1', 'C[C@@H](NCC1CCC(C(N)=O)CC1)c1nccnc1C1CC1', 'ClCc1nc([C@@H]2CCCN(C(=O)[C@@H]3CCCO3)C2)cs1', 'N=CC1(Sc2nc(Cl)ncn2)CCCCC1', 'COc1nc2c3c(sc2n1CCCN(C)C(=O)C[S@@](=O)C)CC3', 'Oc1cccc2ncnc(C(=O)Nc3cccc(C[S@](=O)c4ccccc4F)c3)c12', 'Clc1ccc(Br)c([C@H]2[C@H]3C=C4C[C@@H](CSC4)[C@H]3OC(=O)C[C@@H]2C(=O)[O-])c1', 'OCc1ncsc1OC[C@H]1CN(Cc2ccccc2)C2(CCOCC2)C1', 'Clc1ccccc1-c1cscc1-n1cccn1', 'Clc1ccc(-c2nnnn2-c2cc([N+](=O)[O-])ccc2OC(F)F)cc1', 'O=C(COCc1ccccc1[N+](=O)[O-])Nc1c(F)cccc1F', 'Clc1ccc(Br)cc1N(C(=O)c1ccc(F)cc1)[C@@H]1CSCS1', 'O=C(COc1cc(N)c(C(C)C)cc1[N+](=O)[O-])N1CCCC1', 'CNC(=O)c1cccc([C@@H]2CCCCN2C(=O)c2cc(F)ccc2)c1', 'Oc1cc(C(F)(F)F)nc(CN(Cc2ccccc2)C2CC2)c1', 'OCc1nnc2n1CC[C@H]2O', 'COc1nnnc(NC(=O)CN2CC(=O)Nc3ccccc32)n1', 'Clc1ccc(-c2nonc2N2CCN(Cc3cc(Cl)cs3)CC2)cc1', 'Clc1ccc(Br)cc1Sc1c(F)cc(F)cc1F', 'ClC[C@@H]1CCN(Cc2ccc(F)cc2)C1', 'ClCc1ccnnc1', 'ClCc1ccnnc1', 'Clc1ccc(Br)cc1-n1nnnc1-c1ccccc1', 'CNS(=O)(=O)c1c(OCC)nn(C)c1C', 'OCc1nnsc1NC[C@H](O)c1ccccc1F', 'CCc1cccc(NC(=O)C#CCc2ccccc2)c1', 'CNC(=O)N1CCC(CCNC(=O)NCc2nc3cc(C)ccc3n2CC)CC1', 'Clc1ccc(CN(C(=O)CCC2CC[NH2+]CC2)C(F)(F)F)cc1Cl', 'Nc1[nH]ncc1COc1ccc(Cl)cc1', 'Nc1[nH]nc(NCc2ccccc2)c1Cl', 'Clc1ccccc1OCc1ccn(Cl)n1', 'ClCc1n[nH]c(Nc2ccccc2Cl)n1', 'O=C(c1c[nH]c2nccc(Cl)c12)N1CCC(c2nc3ccccc3[nH]2)CC1', 'O=[N+]([O-])c1ccncc1CCl', 'ClCc1nnn[n-]1', 'O=C(c1ccsc1)c1nn(Cc2ccc(F)cc2)c2c1CCCC2', 'ClC(/N/N=C\\C=C/c1ccccc1O)C(=O)Nc1ccccc1C(F)(F)F', 'Clc1cc(F)c([C@@H](CF)N[C@@H]2CCOC3(CCOCC3)N2)cc1', 'Clc1cc(/C=C2\\c3ccccc3C(=O)N2c2cccs2)ccn1', 'O=C(CN(c1ccc(Cl)cc1)S(=O)(=O)c1cccc(Cl)c1)N1CCCC1', 'Clc1ccccc1-c1csc(COc2ccccc2)n1', 'Clc1ccc(-c2cccs2)nc1CNc1ccccc1', 'CC[C@@H]1c2ccncc2CN1C(=O)c1ccc(NC(=O)C(C)(C)C)cc1', 'COc1nn[nH]c1/C=C/C(=O)N(CC#N)C1CC1', 'O=C(CC1CCN(c2ncccn2)CC1)Nc1ccc(F)c(F)c1', 'C[C@@H](Nc1cccc(C#CN2CCOCC2)c1)c1nnc(-c2ccccc2)n1C', 'Clc1cccc2cc([C@H](C(=O)NCc3cscn3)N3CCOCC3)sc12', 'Clc1cccc(Cn2ncc3c2NC23CC2)c1', 'Oc1cccc(Br)c1Oc1ccccc1F', 'Cc1cc(CC(=O)NCc2nc3ccccc3[nH]2)c2ccccc2n1', 'Clc1cccc([C@@H](SCc2ccccc2)C(F)(F)F)n1', 'Oc1ccc([C@H]2C[NH+]=C(N3CCOCC3)Nc3ncnn32)cc1', 'ClC[C@H](O)COc1cccc(F)c1Cl', 'Clc1cccc2c(N)cccc12', 'O=[N+]([O-])c1ccncc1COc1cccc2c1C(=O)CCC2', 'Oc1ccccc1[C@@H](O)Cn1cc(Br)cn1', 'Clc1cccc(S[C@@H]2CCOc3ccc(Cl)cc32)c1', 'Oc1cccc(OCc2cccc3ccccc23)c1', 'Clc1cccc(Br)c1N1CC[NH+](Cc2cccnc2Cl)CC1', 'COCc1c(C)sc2occ(-c3ccc(OC(F)F)cc3)c12', 'Clc1ccsc1C[NH+]1CCC[C@H]1Cn1cccn1', 'O=C(CSCC(F)(F)F)N1CCC[C@](N2C(=O)c3ccccc3C2)C1', 'OCc1nc(Cl)cc(C2CC2)n1', 'Clc1cccc(C[C@@H](Cc2cccnc2)[NH+]2CCN(C(=O)c3ccccc3)CC2)c1', 'COC[C@H](C)OC(=O)CN1C(=O)c2c(C)cccc2N1c1cccc(Cl)c1', 'OC1C[C@@H]2CC[C@@H]1C[NH+](Cc1ccccc1F)C2', 'Clc1ccsc1C[N-][C@H]1[C@H]2CCO[C@@H]2c2ccccc21', 'O=C(/N=C\\c1cc2ccccc2[nH]1)c1cccc(Cl)c1', 'O=C(c1ccco1)[C@H]1CC(c2ccccc2)=NO1', 'ClC[C@H]1C[C@H]1c1cccc(F)c1Cl', 'Clc1ccc(-c2n[nH]c(Cc3ccccc3)n2)cc1', 'OC[C@@H](Sc1nnc(-c2cccc(F)c2)n1C1CC1)c1ccc2cc[nH]c2c1', 'Clc1ccccc1NCc1ccc(Cl)c(Cl)c1', 'Oc1cc(F)cc(F)c1C(=O)Nc1ccc(C(=O)NC2CC2)cc1[N+](=O)[O-]', 'N#Cc1ccc(CN2CCN(C(=O)COc3ccccc3F)CC2)nc1', 'O=[N+]([O-])c1cc(Cl)cc(N2CCC([NH2+]CC[C@@H]3COCCO3)CC2)c1', 'Clc1ccsc1/CCCn1ccc(=O)c2ccccc21', 'Clc1cc(F)c(CN2CCC[C@@H](Cc3cccc4cccnc34)C2)cc1', 'Oc1ccc(/C=C(/Nc2cccc(F)c2)c2ccccc2)cc1', 'Clc1ccc(Br)cc1-c1ccncc1', 'Clc1ccc(Br)cc1-c1ccncc1', 'Clc1ccc(Br)cc1-c1ccncc1', 'Clc1ccc(-c2cc(Br)ccn2)cc1', 'Clc1cc(Cl)cnc1-c1ccccc1', 'Clc1cc(Br)cc(-c2ccncc2)c1', 'Clc1ccc(Br)cc1OCc1cc(Cl)c2c(c1)OCO2', 'Clc1ccccc1C1CC[C@H](c2ccccc2F)C1', 'Clc1ccccc1OCCn1ncnc1-c1cc(Cl)ccc1O', 'Clc1ccccc1CCNCc1cc(=O)n2ncccc2n1', 'Clc1cc(C[NH2+]C[C@@H]2CCCCO2)c[nH]1', 'Clc1cccc(Br)c1Sc1ccccc1', 'O=[N+]([O-])c1cccc2c1OCCCN2[C@H]1CCS(=O)(=O)C1', 'CNC(=O)[C@H](C)CN(C)C(=O)Nc1ccc(F)cc1Br', 'Clc1ccccc1COCCc1cccc(-c2ccco2)c1', 'Clc1ccc(-c2n[nH]c3ncnn23)cc1Cl', 'N#C[C@@H](c1ccccc1)N1CCC(C[NH2+]C[C@H]2CCOC2)CC1', 'Clc1ccccc1N1NC2(CCCCC2)N1', 'Clc1ccc([C@@H](CSCC(F)(F)F)n2c(CO)nc3ccccc32)cc1', 'Clc1ccsc1C[NH2+][C@@H]1CCc2c(F)cccc21', 'O=C(c1cc(Br)cs1)N1CCC(OCc2ccccc2)CC1', 'Clc1cccc2nc(N3CCCC3)[nH]c12', 'Clc1ccc(Br)c(Oc2ccc(F)cc2)n1', 'Clc1ccsc1SC[C@@H]1CCC[NH2+]C1', 'Clc1ccccc1[C@H](CCl)c1c(F)cccc1F', 'CN(Cc1cccc(Cl)c1)C(=O)c1c[nH]nc1-c1ccc(OC(F)F)cc1', 'Clc1ccc(Br)cc1S(=O)(c1ccccc1)c1ccccc1', 'Clc1ccc(-c2cccs2)nc1CN1CCCc2ccccc21', 'Clc1ccc(C2SCCOC2)nc1', 'Oc1ccccc1CCOc1ccc([N+](=O)[O-])c(Cl)c1', 'O=[N+]([O-])c1cc(Cl)ccc1CCOc1ccccc1F', 'Clc1cc(F)c([C@H](Oc2ccccc2)[C@@]2(C(F)(F)F)NC(=S)N=C2C(=O)[O-])cc1', 'Clc1ccccc1-c1cnccc1N[C@@H]1CCN(c2cccc(Cl)c2)C1=O', 'Clc1ccc(Br)c(SCCn2cncn2)c1', 'Clc1ccsc1NCC[NH+]1CCCC1', 'O=C([O-])[C@H]1[C@@H]2c3cccc(F)c3CC[C@H]1O2', 'COC[C@@H]1CN(C(=O)C2=c3ccccc3=[NH+]C2)N=N1', 'Clc1cccc(F)c1N(C[C@@H]1Oc2ccccc2OC1=O)c1ccccc1', 'Clc1cccc([S@@]c2cc(I)cnc2Cl)c1', 'C=C(C)C[C@@H](NC(=O)CCc1ccc(F)cc1)C1CC1', 'Clc1ccc(Br)cc1C1CC[NH+](Cc2ccc(CO)o2)CC1', 'Clc1cc(CCOc2ccccc2)sc1NCc1ccccc1F', 'Oc1ccc(O[C@H](CNC(=O)c2ccco2)N2CCSCC2)cc1', 'O=[N+]([O-])c1ccncc1S[C@@H]1CC(=O)N(c2ccccc2F)C1', 'N#Cc1c(NC(=O)C=CCCS)c2ccccc2n1C', 'O=[N+]([O-])c1c(Cl)nsc1N/N=C\\c1ccc(-c2cccnc2Cl)cc1', 'COCc1cc(OC)ccc1NC(=O)N(C)Cc1ccc(F)cc1', 'Clc1ccccc1Nc1cnc([O-])nc1C1CC1', 'CN=N[C@]1(C#N)CCC[C@@H]([NH+]2CCN(c3ccccc3F)CC2)C1', 'NS(=O)(=O)Cc1cn2ccccc2[nH+]1', 'O=[N+]([O-])c1c(Cl)nnc2[nH]nc(C3CCCCC3)c12', 'O=[N+]([O-])c1c(Cl)n2nnc(C3CCCCC3)c2[nH]1', 'Clc1nnnc(O[C@@H]2CCCCCC[C@@H]2O)n1', 'Clc1nnnc(NN2CCOCC2)n1', 'Clc1ccc(Br)cc1NC[C@@H](O)c1ccc(Cl)cc1', 'Clc1ccc(Br)c([C@@H](CO)Nc2ccc(Br)cc2)c1', 'Oc1ccc(/C=C/NC(=S)Nc2c(F)cccc2Cl)c2ccccc12', 'Clc1ccc(Cc2cccs2)nc1', 'C=C1c2ccc(OCC(=O)OCC(=O)N/N=C/c3ccccc3)cc2O1', 'Clc1cc(C(F)(F)F)ncc1N1CCCSCC1', 'Oc1cccc2cc(/C=N\\c3ccc(Cl)cn3)ccc12', 'NS(=O)(=O)N[C@H]1CC[NH+]([C@@H]2CCN(c3cccnc3)C2)C1', 'Oc1ccc(/C=N/[C@@H]2CCCOC2)cc1', 'COc1nnnc(Nc2ccc(OC)cc2)c1C', 'Clc1ccccc1Oc1cnc2sn(-c3ccccc3)c2c1', 'Clc1ccc(Br)cc1-n1ncn(CN2CCOCC2)c1=O', 'Clc1ccc([C@@H](OC2CCCC2)C(F)(F)F)o1', 'Cc1ccsc1CNC(=O)[C@H]1CCC(=O)NC1=S', 'Oc1ccc(F)cc1OC[C@@H]1Cc2ccccc2S1', 'Clc1nncc(S(=O)(=O)N[C@@H](Cc2ccccc2)c2ccccc2)c1', 'OCc1n[nH]c(N[C@H]2CC[S@](=O)C(Br)(Cl)C2)n1', 'O=C(/N=c1\\[nH]c2c(ccc3ccccc13)[C@@H]1CCCN12)c1cccc(F)c1', 'Clc1ccsc1/N=C/c1cccc(OC2CCCC2)c1', 'N=C1CN(C(=O)[C@H]2CCCN(C(=O)C3COC3)C2)c2ccccc21', 'Clc1cccc2c(NCc3cccc([N+](=O)[O-])c3)ncnc12', 'Clc1cc(Cl)ncc1N1CCc2ccccc21', 'Clc1ccc(Br)cc1/C=N\\Sc1ccc(C(=O)N2CCCC2)c2nccnc12', 'Cc1cn([C@@H]2CCCCCN2c2ccccc2)c(C#N)c1N', 'Clc1cccc2cc(F)[nH]c12', 'Clc1cccc2cc(F)[nH]c12', 'ClC[C@@H]1CCc2cc(Oc3ccc(Br)cc3)oc21', 'Clc1ccccc1O/N=C/c1ccccc1O', 'Clc1cc(NC[C@@]2(C(=O)[O-])CCCN2S(=O)(=O)c2ccccc2)sc1C1CC1', 'Oc1ccc([C@H]2NC(=O)c3ccccc32)c2ccccc12', 'Clc1cccc([S@@]CCc2ccc(CO)cc2)n1', 'Nc1cn[nH]c1Sc1ccccc1', 'Nc1[nH]ncc1Sc1ccccc1', 'Clc1cc(Cl)nnc1CC[NH2+]Cc1cccc(C(F)(F)F)c1', 'O=C(COc1cc(F)ccc1Cl)N1CCCCCC1', 'Nc1[nH]nc(N[C@@H](c2cccs2)c2cccs2)c1Cl', 'Clc1ccsc1Nc1ncnc(CO)c1-n1cccc1', 'Clc1ccc(-c2n[nH]c(N/N=C(\\[O-])c3ncns3)n2)cc1', 'O[C@@H](c1ccc(Br)o1)C1CCN(S(=O)(=O)c2ccc(F)cc2)CC1', 'Clc1cccc(Br)c1Cn1nnc2cc(F)ccc21', 'OCSc1cn[nH]c1C#N', 'Clc1cccc(F)c1[C@@H]([O-])[C@H](CSc1ccc(F)cc1)N1CCCC1=O', 'Clc1ccccc1Nc1cc(Cl)ccc1OCc1ccccc1F', 'O=C(COc1cc(Cl)c2ccccc2c1O)NNC(=O)COc1ccccc1', 'Clc1ccsc1-c1cscn1', 'Clc1ccsc1-c1cn2ccccc2n1', 'Clc1ccc(-c2csc3scnc32)cc1', 'Clc1ccc(-c2csc3scnc23)cc1', 'Clc1ccc(-c2csc3scnc23)cc1', 'Clc1ccc(-c2csc3sccc23)nc1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'Clc1nnnn1-c1ccccc1', 'Clc1nnnn1-c1ccccc1', 'Oc1ccccc1-n1nnnc1SCCCCCO', 'O=[N+]([O-])c1ccc(/C=C/CCCSc2ccccn2)c([N+](=O)[O-])c1', 'Clc1ccccc1OC1CCC([NH3+])CC1', 'Clc1cc(Br)c(S(=O)(=O)N2CCCC2)cn1', 'Clc1cc(Cl)c(C[NH+]2CCC[C@H]2CN2CCOCC2)cc1Cl', 'O=C(C1CCC1)[C@H]1CCCN(c2cc(C(F)(F)F)ccc2Cl)C1', 'NS(=O)(=O)Cc1cn(Cc2ccccc2)nc1SCc1ccccc1', 'O=C(c1cnc(CN2CCN(c3ncccc3F)CC2)s1)c1ccccc1', 'COc1nc(S(=O)(=O)CC(C)C)ccc1C', 'Clc1ccccc1-c1cncc([C@H]2CCOC2)n1', 'Clc1ccc([C@@H](Br)CCCO)cc1', 'O=C(Cc1cc(=O)n2ccsc2n1)Nc1ccccc1N1CCCC1', 'ClCc1ncoc1Cl', 'O=C(C1CCCC1)N1CCS(=O)(=O)c2ccccc21', 'Clc1ccc(Br)cc1NCCn1cccn1', 'Clc1cc(/N=C\\CCn2ccc(Br)c2)ccn1', 'Clc1cc(C[C@@H]2SCCS2)ncn1', 'NS(=O)(=O)CCN(C(=O)Cc1ccco1)Cc1ccccc1Br', 'Clc1ccc(-c2csc(NC[C@@H]3CCCCO3)n2)cc1', 'Oc1ccc(/C=N\\CCN2CCN(S(=O)(=O)c3ccc4c(c3)OCO4)C[C@H]2c2ccccc2)cc1', 'Oc1ccc(Oc2ccc(C=O)cc2)cc1', 'Clc1ccccc1OC1(Nc2c(F)ccc(Br)c2)c2ccccc2O1', 'C=CC#CCc1cccc(NC(=O)N2CCC(NC(=O)c3ccccc3)CC2)c1', 'Clc1ccc(Cc2nnnn2-c2ccccc2F)cc1', 'N=Cc1nonc1Cc1ccc(Cl)cc1', 'Clc1nncc(OCc2ccccc2F)n1', 'Oc1cccc(-c2csc(CN(c3ccc(Cl)cc3)S(=O)(=O)c3ccccc3)n2)c1', 'Clc1ccc(CCNC[C@@H]2CCOc3ccc(F)cc32)cc1', 'Clc1ccccc1O/Cn1nnc2ccccc2c1=O', 'Clc1ccc(Br)cc1CCc1ccccc1', 'Clc1cc(Br)cc2cnccc12', 'ClCc1csc(-c2cc(Cl)ccc2NCc2ccccc2)n1', 'Oc1ccc(Oc2ncnc3sccc23)c2ccccc12', 'O=C(/C=C/c1cccs1)N1CCN(c2ccc(Cl)cc2Cl)CC1', 'O=[N+]([O-])c1c(F)ccc(CNc2ncccc2F)c1Cl', 'Oc1cccc(SC[C@H]2C[C@H]2c2ccc(Cl)cc2)c1', 'Oc1cc(O)cc2ncnc(N[C@@H](CO)[C@H](O)C(F)(F)F)c12', 'Oc1cccc(O)c1C(=O)N1CCC2(CC2)OCCN1Cc1ccccc1', 'Clc1nnc(S(=O)(=O)NC[C@@H]2CCO[C@@H]2CO)cc1F', 'ClCc1csc(OCC(=O)c2cccc(Cl)c2Cl)n1', 'O=C(Cc1nc(N2CCOCC2)nc2c(C(F)F)ncn12)Nc1ccc([N+](=O)[O-])cc1', 'CCCCO[C@H]1C[C@H](O)CN1Cc1cccc(OC)c1', 'Clc1ccc(Br)c(OCOC2CC[NH2+]CC2)c1', 'Clc1nncc(-n2ccnc2)c1NC1CC1', 'Clc1cccc(/C=C/C[NH2+][C@@H](C(=O)[O-])C2CCCC2)c1', 'O=[N+]([O-])c1cc(C(F)(F)F)ccc1Oc1ccc(Oc2ccccc2)[nH]c1=O', 'O=[N+]([O-])c1ccncc1[C@@H](O)c1ccccc1', 'ClCc1ccnc(-c2noc(-c3ccccc3)n2)n1', 'O=C(COc1cc([N+](=O)[O-])ccc1F)NCC(=O)N1CCOCC1', 'Clc1ccsc1SCC1(Cn2ccnc2)CCOCC1', 'Clc1ccc(-c2nnc(CSC[C@@H]3CCCO3)s2)cc1', 'Clc1ccc(-c2nn(-c3ccc(F)cc3)cn2)cc1', 'Clc1ccc(-c2nn(-c3ccc(F)cc3)cn2)cc1', 'O=C([O-])CSc1ccc2c(c1)CCC(=O)N2', 'Clc1ccccc1N1Nc2c[nH+]ccc2C[C@@H]1c1cc(Cl)ccc1Cl', 'Clc1cccc(-c2n[nH]c(CSc3nncc4ccc(Br)cc34)n2)c1', 'Clc1ccc([C@@H](Cl)Cc2cccs2)cc1', 'O=[N+]([O-])c1cc(Br)c(F)c(/C=N/Nc2cn(C)c3ccccc23)c1', 'Clc1cccc(/N=C/c2cccc(-n3cccn3)c2)c1', 'NS(=O)(=O)NCc1csc(C(=O)N2CCCC[C@@H]2C)n1', 'Clc1ccc(-c2nn(-c3ccc(F)cc3)c([O-])n2)cc1', 'Clc1ccccc1NCC1CCN(S(=O)(=O)c2ccc(Br)o2)CC1', 'O=C(c1cc(Br)cn2cnnc12)N1CC[NH+]2CCCC[C@H]2C1', 'Oc1cccc2cc(-c3cccc(Cl)c3)[nH]c12', 'Nc1c(Cl)nc2ccccc2c1N[C@H]1Cc2ccccc2C1', 'NS(=O)(=O)CCNS(=O)(=O)c1ccccc1', 'O=[N+]([O-])c1ccn(Cc2noc(-c3ccc(F)cc3F)n2)c1', 'O=[N+]([O-])c1c(F)cc(F)cc1NCc1cnn2cnnc2c1', 'ClCc1cccc(OCc2cc(-c3ccccc3)n[nH]2)c1', 'Clc1cc(Cl)ncc1Cc1ccccc1', 'ClCc1cn[nH]c1-c1ccccc1', 'Clc1ccsc1[C@@H](Cl)CCOc1ccccc1F', 'ClCc1nnsc1C(=O)NC1CCCCC1', 'ClCc1csc(O[C@@H]2CCOC2)n1', 'Clc1ccc([C@@H](NNC(=O)c2ccccc2Cl)N2CCCCC2)cc1Cl', 'Nc1[nH]nc2c1[C@@H](c1ccc(Cl)cc1)C1=C(CCCC1=O)C2', 'O=C(C1=CC[C@@H](c2ccccc2)/C(=C\\c2cccs2)C1)N1CCC(O)[C@H](O)C1', 'O=C(Cc1c([O-])c2cnccc2oc1=O)NNC(=O)Cn1ccc(=O)[nH]1', 'Clc1cc(Cl)cn2c(-c3ccco3)cccc2n1', 'Clc1nncc(Cl)c1N1CCN(C(=O)[C@@H]2C=c3ccccc3=[NH+]2)CC1', 'Clc1cccc(F)c1Nc1cccc2ccccc12', 'Clc1ccc(COC[C@@H](O)C[NH+]2CCCCC2)cc1', 'CNS(=O)(=O)c1c(-c2ccccc2)nc(Nc2ccccc2)n1C', 'Clc1ccc([C@@H]([C@@H]2C[NH2+]CCO2)N2CCOCC2)cc1', 'Clc1ccccc1N1Nc2c(cc(N3CCCC3)c(=O)c3ccccc32)CCC1', 'Clc1ccc(C2=C[C@@H](O)[C@]3(CCCN(c4ccccc4)C3)C2)cc1', 'O=C(/C([O-])=N/S(=O)(=O)c1ccccc1Br)c1[nH]ccc1C', 'COc1nnnc(NC(=O)CNc2ccc(Cl)c([N+](=O)[O-])c2)c1C', 'OCc1nc(Cc2ccccc2)no1', 'Nc1nncn1CCc1ccccc1', 'O[C@@H]([C@@H]1C=c2ccccc2=[NH+]1)c1cc([N+](=O)[O-])ccn1', 'COc1nnnc(N2CC[C@@H](Cc3csc(C(=O)NCCCOC)n3)C2)c1C', 'ClCc1n[nH]c(-c2cncc(F)c2)n1', 'Oc1ccc(F)c(Cl)c1CCc1ccc2ccccc2n1', 'CNS(=O)(=O)[C@@H](NCc1cccc(OC)c1)c1ccccc1', 'O=[N+]([O-])c1c(OCO)nc(-c2cccnc2)n1C[C@@H]1CCCO1', 'Clc1cc(F)c(OCc2ccccc2F)cc1C1CC1', 'CN(CCc1cccc(Cl)c1)C(=O)c1cnc([C@@H]2CCCO2)nc1C', 'Clc1cccc(-c2cccc3c2O[C@@H](CN(C(C)C)C2CC3)C2)c1', 'NS(=O)(=O)/N=C\\c1ccc(OCc2ccccc2)cc1', 'Clc1ccccc1Oc1cc([N+](=O)n2cc(Br)cn2)c[nH]1', 'Clc1ccc(Br)cc1-n1cccc1Cn1ccccc1=O', 'Oc1cccc2nnccc1c1ccccc12', 'Oc1cccc2ncoc12', 'Oc1cccc2ncoc12', 'Oc1cccc2nccnc12', 'Clc1ccc(-c2nnnn2-c2ccccc2Br)cc1F', 'COc1nnnn1[C@@H]1CC(C)(C)CCN1C(=O)c1cc(F)ccc1F', 'Clc1cc(Cl)nc(SCc2noc(C3CC3)n2)n1', 'Clc1ccccc1Oc1cc(Br)c[nH]1', 'O=C(/N=C/CCSc1ccccc1)c1ccco1', 'O=[N+]([O-])c1c(O[C@@H]2CCCCCC2=O)cc(N2CCOCC2)c2ccccc12', 'Clc1cccc(N2CCC/C(=C/c3cccc(C(F)(F)F)c3)SC2=O)c1', 'Oc1ccc(/N=c2/[nH]c3ccccc3[nH]2)cc1N1CCCC1', 'ClC[C@@H]1C[C@H]1c1cccc(Cl)c1', 'O=C(c1cn[nH]n1)N1CCC(Oc2ccccc2)CC1', 'Nc1[nH]ncc1C(=O)O[C@@H]1CCCC[C@H]1c1ccccc1', 'N=CCNC(=O)c1cncc2c1CCO2', 'NS(=O)(=O)C[C@H](NC(=O)Cc1csc(Nc2ccc(F)cc2)n1)c1ccccc1', 'O=[N+]([O-])c1c(Cl)nnc(NC2(C#N)CCOCC2)c(-c2ccc(Cl)cc2)c1=O', 'O=C(/C=C\\C[N-]c1ccc(Cl)cc1Cl)C1CCCCC1', 'N#Cc1ccccc1O[C@@H](C)C(=O)N1CCOC[C@@H]1C1([NH+](C)C)CCCCC1', 'Nc1[nH]nc(CNc2ccc3c(c2)CCCC3)c1[N+](=O)[O-]', 'Cc1ccsc1[C@H]1[NH+]=c2ccccc2=C1N', 'O=[N+]([O-])c1c(OC[C@H](O)COc2cccc(F)c2)nccn1', 'Clc1ccc(Br)c(/C=N/c2c[nH]c3ccccc23)c1', 'ClCc1n[nH]c2nccc(-c3ocnc3)c12', 'O=C(COCC(F)(F)F)NOCc1cccs1', 'CN(CC(F)(F)c1ccc(Cl)cc1Cl)C(=O)N[C@@H]1CCC[C@H](C)C1', 'OCc1nncn1Cc1cccc(F)c1F', 'Clc1ccc(C2SCCC[S@@]2=O)cc1', 'O=C(c1cc(Br)c[nH]1)C1CCCCC1', 'ClCc1cncc(C(=O)OCc2nc[nH]n2)c1', 'Oc1cc(O)cc(-c2ncccc2OCc2ccccc2)c1', 'Clc1cccc(Br)c1C[C@H](O)c1cnccn1', 'Oc1ccc(/N=c2/[nH]c(C(F)(F)F)ccc2-c2cccc(Br)c2)cc1[N+](=O)[O-]', 'Clc1ccccc1Oc1cc(-c2nc3c([nH]c2=O)CCCC3)ccc1O', 'Clc1cc(Br)cn1NC1CCCCC1', 'Clc1ccc(-c2nonc2[S-])cc1', 'Clc1ccc(-c2nonc2[S-])cc1', 'Clc1ccc(-c2nonc2[S-])cc1', 'Clc1ccc(-c2nonc2[S-])cc1', 'Clc1ccc(CNc2cccc(Cl)c2)cc1', 'NS(=O)(=O)C[C@H]1C[C@@H]1c1ccccc1', 'Clc1ccc(-c2ccccc2Cl)nc1C1CC1', 'OCc1n[nH]cc1Oc1ccc(Br)cc1', 'O=C(C1CC1)N1CCCN(Cc2cccc(Cl)c2)CC1', 'Clc1cccc([S@@](=O)[C@@H]2CC[C@H](Nc3ccccc3F)C2)c1', 'OCc1n[nH]c(O)c1-c1ccccc1', 'ClCc1ncnn1-c1ccccc1', 'ClCc1ncn(-c2ccccc2)n1', 'ClCc1cnn(-c2ccccc2)c1N', 'Clc1cc(Br)cc2cccnc12', 'Clc1cc(Br)cc2cccnc12', 'Clc1cc(C2(C(=O)[O-])CCCCCC2)no1', 'COc1ncnc(N)c1CN1CCN(c2ccc(F)c(F)c2)CC1', 'O=[N+]([O-])c1c(OCOC[C@H]2CCCO2)c(Cl)ccc1F', 'OCc1nc([O-])c(-c2ccccc2)o1', 'Clc1ccsc1-c1nonc1NCc1ncc[nH]1', 'CNC(=O)c1ccc(NC(=O)CNC(=O)c2ccccc2Br)cn1', 'Clc1ccsc1S(=O)(=O)Nc1ccc(C(=O)N2CCOCC2)cc1', 'O=C(Cc1noc(-c2ccncc2)n1)N[C@@H]1CCS(=O)(=O)C1', 'Clc1cccc2ccc(S(=O)(=O)NCC(c3ccccc3)c3ccc(Cl)cc3)nc21', 'Clc1cc(F)cc([C@H]2CCCN2CCn2cnnc2)c1', 'Clc1ccsc1Cn1c(=O)n(Cc2ccco2)c2ncccc21', 'Oc1ccccc1-c1ccc(Cl)cc1', 'Clc1cc(C[C@@H]2Cc3ccccc3S2)[nH]c1NCc1cc(F)ccc1F', 'O=C(c1ccsc1)c1c(O)ccnc1SCC(F)F', 'Clc1ccsc1SC[C@H]1C[C@@]12OCCC[NH2+]C2', 'Clc1ccc(-c2cccc(C[NH2+]COc3ccc(F)cc3)c2)cc1', 'ClC(F)(CO)[NH2+]Cc1ccc(Cl)cc1', 'Clc1nnnn1c1c(Cl)cccc1N1CC[NH2+]CC1', 'NS(=O)(=O)/N=C(/c1ccccc1)[C@@H]1CCCN(c2ccc(Cl)cc2)C1', 'Oc1ccc(F)c([C@@H]2CCC[NH+]2Cc2ccccc2CO)c1', 'OCSCCOCCc1n[nH]c2cc(OC(F)F)ccc12', 'CN[C@H](c1ccccc1)c1ccc(F)cc1Cl', 'Clc1ccc(Br)c(OC[C@@H]2C=c3ccccc3=[NH+]2)c1', 'Clc1ccccc1O/N=C(/NCc1cncc(C(F)(F)F)c1)N1CCOCC1', 'O=C([C@H](SCc1cccs1)N1C(=O)CSC1=O)Nc1ccc(Br)cc1', 'Clc1nncc(F)c1-c1ccc(F)cc1', 'Clc1ccccc1-c1nncc(Cl)c1F', 'Clc1ccc(-c2n[nH]cc2Br)nc1', 'Clc1cc(Cl)c2c(c1)CCN2C1CCC(CN2CCOCC2)CC1', 'Clc1ccccc1-c1cncc2c1CN2C(F)(F)F', 'Clc1ccccc1-c1ncc(S(=O)(=O)N2CCC[C@H]2C(N)=O)cn1', 'Clc1nnnc(NN=CSc2ccc(Cl)cc2)n1', 'Clc1ccc([C@@H](Nc2nccs2)C2CC2)cc1', 'ClC(/C=N/c1cccc(Br)c1)c1ccccc1', 'Clc1cccc2[nH]nc(NC[C@H](O)c3ccccc3)c12', 'Clc1cccc2[nH]nc(NC[C@@H](O)c3ccccc3)c12', 'Clc1cccc([C@@H]2[C@@H](c3ccc(Cc4cnn[nH]4)nc3)N=C2Cl)c1', 'OC[C@H](CNS(=O)(=O)c1cccs1)c1ccccc1', 'Clc1ccccc1Cc1c(Cl)nc(Cl)nc1Cl', 'O=[N+]([O-])c1cc(O[C@@H]2CCCC[C@H]2[C@@H]2CCC[NH2+]C2)c(Cl)cc1Cl', 'Clc1cccc(CN2CCCC[C@H]2c2nc3ccccc3o2)c1', 'Clc1cc(CCc2c[nH]c3ccccc23)[nH]c1', 'Clc1ccc(Br)cc1-c1cnc2ccccc2n1', 'O=C(c1cn2ccsc2n1)N1CCC[C@H](CSc2ccncc2)C1', 'Clc1cccc(/N=C2\\CCCCCc3nc[nH]c32)c1', 'Oc1ccccc1OC[C@H](O)c1ccc(Cl)c(Cl)c1', 'O=C(COCCN1CCCC1=O)Nc1cc(Br)cc(Cl)c1', 'Clc1ccc(CNC[C@@H]2CC[C@@H](C(F)(F)F)O2)cc1[N+](=O)[O-]', 'OC[C@@H](Cc1cc(F)cc(Br)c1)OCC(F)(F)F', 'Clc1ccsc1SCC(=O)N1CCC[C@H]1c1ccc2c(c1)OCO2', 'Clc1ccccc1-c1n[nH]c2cnn(Cc3cccs3)c12', 'O=C(c1cn2ccsc2c1N1C[C@@H]2CCCC[C@H]2C1)Nc1cccc2cccnc12', 'Clc1cccc(/N=N/OCC[NH+]2CCCCCC2)c1', 'O=C(/N=c1c(N2CCN(c3ccccc3)CC2)nc2cc3c(cc12)CCCC3)c1ccco1', 'O=C(Cc1c(-c2ccc(Cl)cc2)cccc1O)Nc1ccc(C(F)(F)F)cc1', 'C=CCC1CC[NH+](Cc2cnc3[nH]c[nH+]c3c2)CC1', 'O=[N+]([O-])c1c(Br)cc(S(=O)(=O)N2CCOCC2)cc1OC(F)F', 'Clc1cccc(F)c1N[C@H]1CC(Cc2ccccc2F)=[NH+]1', 'Cc1c([N+](=O)[O-])cc(C#N)c(C)c1SCC(=O)NCc1ccccc1', 'O=C(c1cc(F)ccc1F)N(CCCN1CCOCC1)S(=O)(=O)c1cccc(Cl)c1', 'ClCc1ncnn1-c1ncnc2c1CCCC2', 'OCc1ncsc1Nc1ccccc1C(=O)N1CCCC1', 'O=C(/C=C\\c1cccs1)NC1CCN(c2ncccn2)CC1', 'Oc1cc(C[NH+]2CCSC[C@@H]2C2CCCC2)cs1', 'Clc1ccccc1Nc1nnnn1-c1ccccc1', 'Clc1cccc(Br)c1OC[C@H](O)COc1ccc2nc[nH]c2c1', 'O=[N+]([O-])c1ccnc2c1[nH]c(=S)n2C1CCCCCC1', 'OC[C@@H]([NH2+][C@H]1CCOC2(CCCCC2)C1)c1ccccc1', 'O=C([C@H](Nc1ncccc1Br)C1CC1)Nc1nccs1', 'O=[N+]([O-])c1c(F)cccc1CN1CCCCC[C@H]1c1cccc(F)c1', 'Oc1ccc(O[C@H](Cc2cscn2)C(=O)c2cccc3ccccc23)cn1', 'O=C(CN(Cc1ccccc1)c1ccccc1)Nc1cc(Cl)ccc1Cl', 'Clc1ccsc1/C=[NH+]/C[NH+]1CCC2(CCOCC2)C1', 'Oc1ccccc1/N=N\\c1ccccc1', 'C=C1c2c(cccc2C)[C@H]1CC(=O)N[C@@H]1CCc2cc(SC)ccc21', 'ClC(F)(F)C1CCN(Cc2csc(-c3ccc(F)cc3)n2)CC1', 'C=C1[C@H]2CC[C@@H]1CN(Cc1nc3ccccc3s1)CC2', 'Clc1ccsc1[C@H](Cl)[C@@H]1CCOC2(CCCC2)C1', 'Clc1ccc([C@@H]([O-])N/N=C/c2ccco2)cc1', 'Clc1cccc([S@@]C(Br)(CO)C(F)(F)F)c1', 'CN(Cc1ccc([N+](=O)[O-])cc1)C(=O)NCc1ncn(C)n1', 'CC[C@@H](C#N)Oc1ccc(NC(=O)Cn2cc([N+](=O)[O-])cn2)cc1', 'Oc1ccc([N+](=O)Cc2nnnn2-c2ccccc2)c(Cl)c1', 'O=C(c1cc([N+](=O)[O-])ccc1-n1cccn1)N1CC1(C#N)C1CC1', 'Oc1cccc2cc(OC[C@H]3CCCO3)ccc12', 'Clc1ccc(Br)cc1[C@@H](c1ccccc1)S(=O)(=O)c1cccc([N+](=O)[O-])c1', 'O=[N+]([O-])c1c(Cl)nsc1N[C@@H]1CC[NH+]2CCCC[C@@H]12', 'Oc1ccc(/N=c2\\sc(Cl)cc(Cl)c2C[NH+](CC(=O)[O-])c2ccccc2)cc1', 'O=C(/C=C/c1ccc(COc2ccc(Cl)cc2)o1)Nc1cc(Cl)ccc1Cl', 'C=C1NN(C(=O)c2cnccc2)[C@H]2N=CN=C2[C@H]1C', 'O=C(Cc1c(Cl)n[nH]c1Cl)NCc1cccc(Cl)c1', 'Oc1cccc([C@H](COC2CC[NH2+]CC2)N2CCOCC2)c1', 'CN=C/c1cc(-c2ccccc2)nc(S)c1C', 'Oc1ccccc1-c1nc(-c2csc(NC3CCC3)c2)no1', 'Clc1ccc(-c2cc(Br)ccc2N2CCOCC2)cn1', 'Clc1ccc(-c2n[nH]cc2N2CCCC2)nc1[N+](=O)[O-]', 'O=[N+]([O-])c1ccc[nH+]c1NCCSc1ncccn1', 'ClCc1csc(-c2ccccc2)n1', 'ClCc1csc(-c2ccccc2)n1', 'ClCc1csc(-c2ccccc2)n1', 'ClCc1csc(-c2ccccc2)n1', 'Oc1cccc(O[C@@H]2[C@H](O)C(C(=O)[O-])CCS(=O)(=O)C2)c1', 'N=C1N2CCS(=O)(=O)C[C@@H]2[NH2+]Cc2ccccc21', 'Clc1ccc(-c2n[nH]c3c2[C@H](c2ccccc2)NC3=O)c(O)c1Cl', 'Clc1ccc(Br)cc1-c1cn(-c2ccccc2F)[nH]c1=O', 'O=C(CS(=O)(=O)c1ccc2ccccc2c1)Nc1ccccc1', 'O=C(CNS(=O)(=O)c1ccc(F)cc1Cl)NCc1ccccc1', 'Clc1ccc(-c2n[nH]c([O-])n2)cc1', 'ClCc1cccc(Cl)n1', 'OCc1nc2ncn(-c3ccc(Br)cc3)c2c(Cl)c1Cl', 'Oc1cccc(Sc2nnnn2-c2c(C(=O)[O-])cnn2-c2ccc(Cl)cc2)c1', 'O=C(CNS(=S)=N/O)c1ccccc1', 'COc1nc2nc(N(C[C@@H]3CCCO3)C(C)C)c(C)nc1-2', 'Clc1ccc(Br)cc1N(Cc1ccns1)C(=O)c1cccc(Br)c1', 'Clc1ccsc1[C@H]1COc2ccccc21', 'O=[N+]([O-])c1c(-c2cc(-c3ccccc3)on2)sc2ncccc12', 'O=C(CNS(=S)(=O)c1ccccc1Cl)NCC(F)(F)F', 'O=C(CCNS(=O)(=O)c1ccccc1)Nc1ccccc1', 'O[C@@H]1[C@@H](Cc2ccccc2)N/C1=C[C@@H]2c1ccccc1C2=O', 'N=CCc1nnc(C(=O)N(C)Cc2ccccc2OCC(F)F)s1', 'Clc1ccccc1OCc1ccn([C@@H]2CCSC2)c1', 'Clc1ccc(Br)cc1COC(=O)[C@H]1C[C@H]1c1ccc(Cl)cc1', 'Clc1cccc2c(N[C@@H]3CCOC3Cl)ncnc12', 'Clc1ccc(-c2nonc2NC(=O)Cc2c(Cl)cccc2Cl)cc1', 'Clc1ccccc1NC(=S)NCc1ccc2c(c1)OCCO2', 'Clc1cccc(NCC[NH+]2CCC[C@@H]2CN2CCOCC2)c1', 'Clc1ccc(-c2csc3[nH]cc(C(F)(F)F)c-23)cc1', 'Clc1ccc(Br)cc1-n1nnnc1SCc1ncc(-c2ccccc2)o1', 'Clc1ccccc1-c1cscc1-n1ccnc1', 'Clc1cccc(/C=N/NC(=S)NCc2ccccc2)c1', 'CNC(=O)Cn1ccc(=O)n(C)c1=O', 'ClCc1cn[nH]c1[C@H]1CCCN1c1ccc2c(c1)OCO2', 'Oc1cc(-n2cn/cn2)ccc1-c1ccccc1', 'Oc1cc(O)cc(Oc2ccc(C(=O)NC3CC3)cc2Cl)c1', 'N=Cc1nonc1CSCc1ccccc1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1-c1ccc(F)cc1', 'Clc1ccc(-c2nnnn2-c2ccccc2)cc1Cl', 'O=[N+]([O-])c1cc(C2CCCC2)nc2nnnn12', 'Clc1cc(Cl)nc([C@H]2CCC[NH2+]2)n1', 'OCc1ncco1', 'Nc1nncn1CCCc1nc(-c2ccccc2F)cs1', 'Nc1c(Oc2[nH+]cccc2F)[nH]c2c1-c1ccc(Br)cc1NC2', 'Nc1c(CSc2ccccc2Cl)cnn1-c1ccc(Cl)cc1', 'O[C@@H]1C[C@@](Oc2ccccc2)c2ccccc21', 'Clc1ccc(-c2n[nH]cc2-n2ccc([N+](=O)[O-])c2)cc1', 'O=C(COC(=O)c1cnc(Br)cn1)Nc1cc(Br)ccc1Cl', 'Oc1ccc([N+](=O)CN2C(=O)COc3ccccc3[C@@H]2Nc2ccccc2)cc1', 'O=[N+]([O-])c1c(F)cc(Br)cc1NC[C@H](O)C1CCC1', 'COc1nc(C2(NC(=O)c3cc(C(C)C)no3)CCCC2)cs1', 'O=C(c1cn[nH]c1-c1ccc(Cl)cc1)N1CCN(C(=O)N[C@@H]2CCSc3ccccc32)CC1', 'COc1nc(N)cc(C(=O)N2CCC[C@@H](CO)C2)c1N', 'O=C([C@H](Cl)c1ccccc1)N1CCOCC1', 'Clc1ccc(Br)cc1N(Cc1cccc(O)c1)c1cccc[n+]1[O-]', 'Clc1cccc([S@@](=O)Cc2cccc(I)c2)c1', 'COC[C@]1(C)CCCN(C(=O)C(=O)Nc2ccccc2)CC1', 'Clc1ccccc1OC1(Cl)CCOCC1', 'ClC[C@H]1OCCc2ccccc21', 'Clc1nncnc1C[NH+]1CCOCC1', 'Oc1cc(O[C@@H](C(=O)[O-])c2cc(=O)[nH]c3ccccc23)ccc1O', 'Oc1ccc(O[C@@H](C2CCCC2)C(F)(F)F)cc1', 'Nc1nncn1/N=N/N=C/c1ccc([N+](=O)[O-])cc1', 'Clc1ccc(C(F)(F)F)cn1', 'Nc1[nH]ncc1C[NH3+]', 'Oc1ccccc1C1OCOc2cc(S(=O)(=O)NCc3ccccc3)ccc21', 'Clc1ccc(-c2nonc2N2CCOCC2)cc1', 'Clc1ccc(-c2cc[nH]n2)nc1[C@H]1COCCO1', 'O=[N+]([O-])c1ccncc1Cn1cccn1', 'Nc1nonc1COc1ccc(Br)cc1', 'N=Cc1nonc1Cc1ccccc1F', 'Clc1cccc(NCC2(O)CCOCC2)c1', 'Clc1ccc(Br)cc1-c1cnc2n1CCN(Cc1cccc([N+](=O)[O-])c1)C2', 'Clc1cc(COCc2ccc[nH]2)ncn1', 'Clc1ccccc1-c1nc(N2CCCC2)nc2ccccc12', 'O=[N+]([O-])c1c(-c2ccccc2)n(-c2ccccc2F)nc1[O-]', 'OCc1ncn(-c2ccccc2)n1', 'Clc1cc(N2CCC[C@H]2c2nnc(-c3ccccc3)o2)ccc1Cl', 'O=C(CNc1cnn(Cc2ccccc2)c(=O)c1Cl)Oc1ccc([N+](=O)[O-])cc1', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n', 'Clc1ccccc1CI\n\n\n', 'Clc1ccccc1CI\n\n', 'Clc1ccccc1CI\n', 'Clc1ccccc1CI', 'Clc1ccccc1CI', 'Clc1ccc(-c2cc(F)c(C[NH2+]Cc3cccs3)cc2)cc1', 'Clc1ccccc1-c1cncc(Br)c1', 'Clc1ccc(CO)c(/C(Cl)=[N+]([O-])/C=C/c2ccccc2)c1', 'Oc1ccc(F)c([C@@H]2CSCCN2C(=O)C2=CCCCC2)c1', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n', 'Clc1ccc(Br)cc1\n\n', 'Clc1ccc(Br)cc1\n', 'Clc1ccc(Br)cc1', 'O=C(CNCc1cccs1)Nc1ccc(Cl)c(Cl)c1', 'Nc1c(F)cccc1N1CC[NH2+]CC1', 'Clc1ccc(-c2nn(-c3cc([N+](=O)[O-])ccc3Cl)nn2)cc1', 'O=C(c1c[nH]c(N/N=C/c2cccc(Cl)c2F)n1)OCc1ccncc1', 'Clc1ccccc1OCc1csc(-c2ccc(Cl)cc2)n1', 'Clc1cccc2[nH]nc(S[C@H]3CC[NH2+]C[C@@H]3Cc3ccccc3)c12', 'Clc1ccc(Br)cc1OCCCOCC(F)(F)F', 'Clc1ccc(-c2cc(Oc3ccccc3)nc(SC3CC3)n2)cc1', 'Clc1ccccc1C1=CCN(c2ccc(F)cc2)CC1', 'CN(CC(=O)N[C@H]1CCN(c2ccccc2)C1=O)C(C)C', 'ClC[C@H]1CO[C@H](N2CC[NH+](Cc3ccco3)C2)CC1', 'Clc1ccc(-c2nonc2)nc1-c1ccc(Br)cc1', 'CC1=C(C(=O)Nc2ccccc2)c2ccccc2C(=O)N1C', 'Oc1cc(-n2cc[nH+]c2)ccc1/N=C/c1ccccc1[N+](=O)[O-]', 'Clc1ccccc1[C@H](Cl)c1ccc(F)cc1', 'O=C(c1c[nH]cn1)O/N=C/C(Cl)=C/c1ccc([N+](=O)[O-])cc1', 'ClCc1n[nH]c(-c2ccccc2)c1N1CCc2cc(F)ccc2C1', 'Clc1cccc2ncnc(NC[C@@H](O)c3ccoc3)c12', 'CN=C/[C@@H]1c2ccccc2C(=O)N1c1ncccn1', 'Clc1ccccc1NS(=O)(=O)c1ccc(C(=O)N2CCOCC2)cc1', 'Clc1ccccc1NCc1c[nH]nc1Cc1cccc(Br)c1', 'O=[N+]([O-])c1cccc2[nH]cn/c(=N/n3c4ccccc4nc3c1)OCOC2', 'O=[N+]([O-])c1c([O-])n[nH]c1NCc1cc2ccccc2o1', 'Clc1ccccc1-c1cscc1C(=O)N1CCc2c([nH]c3ccccc23)C1', 'Clc1cc(F)c(C(=O)N2CCc3nnccc32)cc1F', 'ClCc1nc([C@@H]2CCC[NH2+]C2)cs1', 'O[C@@H](c1cc(Cl)ccc1O)c1c[nH]nc1C1CCCCC1', 'Clc1ccccc1NCc1csc(N2CCCCC2)n1', 'Clc1ccccc1[C@H]1CCC[NH+](Cc2cccc(O)c2)C1', 'Nc1n[nH]c2c1[C@@H](c1ccccc1)CN(c1ccccc1Cl)C2', 'NS(=O)(=O)Cc1noc(CC[S@@](=O)Cc2ccccc2)n1', 'Cc1snc(-c2ccco2)c1CNC(=O)/C=C/c1cccc(OC)c1', 'Oc1cc(-n2cc(/C=C3/C(=N/c4ccccc4)c4ccccc4S3)C2=O)ccc1O', 'Clc1nncc(F)c1N[C@H]1CCc2c(Br)cccc21', 'Oc1ccc(O[C@@H]2C[C@@H](c3cccc(COC(F)(F)F)c3)C2)cc1', 'Oc1cc(C(F)(F)F)cc2c1ncn2-c1ccc(F)cc1', 'Clc1ccsc1SC[C@H]1Cc2ccccc2S1', 'O=[N+]([O-])c1ccnc2cccc(-c3nc4ccccc4s3)c12', 'Clc1ccccc1NCc1cc(F)ccc1Cl', 'Clc1cccc(F)c1NCc1cccc(F)c1', 'Clc1cc(F)cc(NCc2cccc(Cl)c2)c1', 'Clc1ccsc1S(=O)(=O)Nc1ccccc1C(=O)c1ccc(Br)cc1', 'Cc1sc(=O)n(CCC(=O)N[C@H]2CCSC2)c1C', 'ClC[C@@H](CO)c1cccs1', 'Oc1cccc(NCc2ccc(OCc3cccnc3)cc2)c1Cl', 'CN[C@H]1Nn2c(-c3ccc(F)cc3)cnc2N=C[C@@H]1SC', 'O=C(c1cn[nH]c1)N(Cc1cccc2ccccc12)[C@@H]1CCS(=O)(=O)C1', 'Clc1ccccc1CNC(=O)N1CCCc2ccccc21', 'Nc1c(Oc2ccc(F)cc2)nnn1-c1ccc(F)cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(SCc2ccccn2)c1-c1ccncc1Cl', 'Clc1cc(NC(c2ccc3c(c2)OCCO3)[NH+]2CCCCCC2)ccc1Cl', 'Clc1cc(N2CCN(Cc3cccc(F)c3)CC2)ccn1', 'O=[N+]([O-])c1cc(N2CCC(c3cc[nH]c3)CC2)ccc1OCC(F)(F)F', 'O=C(COc1cc(C(F)(F)F)ccc1Cl)N(Cc1ccccc1)S(=O)(=O)c1ccccc1', 'C[C@@H]([NH2+][C@@H]1CC[C@@H]([NH+](C)C)C1)c1cccc(Cl)c1', 'Clc1cc(Br)cnc1I', 'Oc1cccc(O[C@H]2CCN(c3ncccn3)c3ccccc32)c1', 'O=C(CN(c1cn[nH]n1)S(=O)(=O)c1cccc(Br)c1)N1CCN(c2ccccc2)CC1', 'Clc1ccsc1S(=O)(=O)OC1CCC(c2nccn2Cc2ccccc2)CC1', 'N[C@@H]1[C@H](C2CC2)N=C(NCc2cc(Cl)ccc2-n2ccnc2)C1', 'Clc1ccc(-c2cccnc2N=C2CCOCC2)cc1', 'Oc1cc(O[C@@H](C2CCCC2)N2CCCC2=O)cc(Br)c1Cl', 'Oc1ccc(/N=c2/[nH]cc[nH]2)nc1Cl', 'O=[N+]([O-])c1ccnc2ccc(Cl)c(F)c12', 'O=C(c1cnn(Br)c1)c1ccc(Cl)cc1', 'N=Cc1cnc(Cl)nc1-c1ccc(Cl)cc1', 'Clc1nncc(-c2ccccc2Cl)n1', 'Clc1nncc(-c2ccc(F)cc2)n1', 'Clc1nncc(-c2ccc(F)cc2)n1', 'Clc1ccccc1Oc1cncc(Cl)n1', 'Clc1ccccc1Oc1cnc(Cl)cn1', 'Clc1ccccc1Oc1cnc(Br)cn1', 'Clc1ccccc1Nc1ncc(Br)cn1', 'Clc1ccccc1Nc1cnc(Cl)nc1', 'Clc1ccc(Br)c(F)c1-n1cncn1', 'Clc1ccc(-c2cc[nH]n2)nc1O', 'ClCc1csc(S(=O)(=O)N(Cc2ccccc2)[C@H](CO)c2ccccc2Cl)c1', 'Clc1ccc(Br)cc1COc1ccc(Br)cc1', 'Nc1[nH]ncc1[C@@H]([N+](=O)[O-])C(F)(F)F', 'Clc1ccc(COc2ccc(Br)cc2)cc1', 'Oc1cc(F)cc2ncccc12', 'Oc1cc(CCOc2ccccc2F)c2cccnc2n1', 'ClC(=O)c1ccc(N[C@@H]2CSCCS2)cc1', 'Clc1cc(Br)cc2oc(-c3ccc(OC4CCCCC4)cc3)nc12', 'Clc1ccccc1-c1n[nH]c2c1CCCC2', 'O=C(c1cnc(NSc2ncccn2)nc1Cl)[C@H]1CCCO1', 'O=C(CCc1n[nH]c(=O)c2ccccc12)NNC(=O)c1ccc(F)cc1F', 'Nc1nncn1c1ccc(Cl)cc1', 'O=C(c1cc(Cl)ccc1[N+](=O)[O-])N1CCN(S(=O)(=O)c2cccc([N+](=O)[O-])c2)CC1', 'Clc1ccccc1CS(=O)(=O)CC(=O)Nc1ccc(Br)cc1', 'Clc1ccccc1Oc1cncc([O-])n1', 'O[C@@H](Cc1cccc(Br)c1)[C@@H]1CCCO1', 'Clc1ccccc1Nc1ccc2nnc(-c3cccs3)n2n1', 'Oc1cccc(-c2cc(CCOc3cccc4ccccc34)no2)c1', 'Clc1nnn(Cc2ccccc2)c1I', 'C=CC#CCN(C)C(=O)C1CCN(C(=O)c2cccc(F)c2)CC1', 'O=C(c1cc(F)c2c3ccccc3[nH]c(=O)n12)N1CCOCC1', 'O=C(c1cn[nH]c1C(=O)c1ccccc1)N1CCOCC1', 'Clc1ccccc1C(=O)Nc1cnc2ccccc2c1O', 'Clc1ccccc1NC1(c2ncc[nH+]c2Cl)CCOCC1', 'O=[N+]([O-])c1c(Cl)nsc1Nc1ccc([N+](=O)[O-])c([N+](=O)[O-])c1', 'Cc1ccc2nc(C)c(C(=O)OCC(=O)Nc3nc4ccc(Cl)cc4s3)nc2c1', 'Clc1ccc(Cc2cccc(CNC(=O)C3CC3)c2)cc1', 'Oc1ccc(O[C@@H](Cc2ccc(Cl)cc2)c2cccc(Cl)c2)cc1F', 'OC[C@@H](C1CCSCC1)N1CCC[C@@H](c2cc(-c3ccccc3)no2)C1', 'Clc1ccccc1Nc1ncccc1Cl', 'Clc1ccc(-c2cccnc2Cl)nc1', 'Clc1ccc(-c2cccnc2Cl)cn1', 'Clc1ccccc1CCCSc1nc2ccccc2[nH]1', 'Clc1cccc([S@@]CC=N/c2ccc(Cl)cc2Cl)c1', 'O=C(c1cc(Cl)c(Cl)c(Cl)c1)N1CCN(c2cccc[nH+]2)CC1', 'N=CCCOc1ccc(O[C@H]2N=NC(=O)[C@@H]2C)cc1Cl', 'OCc1nn(SC[C@@H]2CCCO2)c2ccccc12', 'Clc1cccc(CN(Cc2nc3ccccc3s2)C2CC2)c1', 'OC[C@@H](NS(=O)(=O)c1ccc(Cl)cc1)c1cccc(F)c1', 'Clc1cc(CCl)nc2ccccc12', 'COC[C@]1(C)CCN(C(=O)c2cc(C)nn2C2CCOCC2)C1', 'O=[N+]([O-])c1c(Cl)n2nnc(-c3ccncc3)c2cc1Cl', 'ClCc1csc(S[C@@H](CO)c2ccc(F)c(F)c2)n1', 'NS(=O)(=O)/N=C/C=C/c1ccc(Cl)cc1OCc1ccccc1', 'O=C(c1c[nH]c2nc[nH]c12)c1ccc(Br)cc1', 'O=[N+]([O-])c1ccc(CSc2nc(-c3ccc(Cl)cc3Cl)ncn2)cc1', 'O=C(c1cc(Cl)ccn1)N1CC[NH2+]CC1', 'Clc1ccccc1C(=O)[C@@H]1CCCC[C@H]1O', 'O=C(COc1ccc(NC(=O)C[C@H]2CSc3ccccc32)[nH]1)Nc1ccccc1', 'Clc1cccc(S(=O)(=O)Nc2ccc(C[NH+]3CCCC3)cc2)c1', 'Clc1ccccc1Nc1ncc([N+](=O)[O-])cn1', 'Clc1ccc(-c2n[nH]c(Nc3ncccn3)n2)cc1', 'O=[N+]([O-])c1cc(N[C@H](c2cccc(F)c2)C2CCCC2)c[nH+]c1', 'Oc1cc(F)nc(SCC(=O)Nc2ccc(S(=O)(=O)C(F)F)cc2)c1', 'ClCc1noc(-c2ccco2)n1', 'Clc1ccc(-c2n[nH]c3ccccc23)[nH]1', 'O[C@@H](C#Cc1cscn1)C1CC1', 'O=C(/C=C/OCc1ccncc1)c1c(Cl)cccc1Cl', 'Clc1nncc(Oc2nc(-c3ccccc3)no2)c1I', 'Oc1ccccc1-c1noc([C@@H]2CCO[C@]3(CCSC3)C2)n1', 'Clc1ccccc1OCc1cscc1Br', 'Oc1cc(-n2nnnc2S[C@@H]2CCCC[C@@H]2C[NH2+]Cc2ccccc2)ccc1Cl', 'ClCc1csc(Cc2cccs2)n1', 'Clc1cc(Br)c(CNc2ccc3c(c2)CCCO3)cc1Cl', 'O=C([O-])/C(=N/c1ccc2c(c1)OCO2)N1CCC(Oc2ccccc2)CC1', 'Oc1ccc(O[C@H]2C=[NH+][C@@H](COc3cc(C(F)(F)F)ccc3Cl)C2)cc1', 'O[C@@H]([C@@H](c1ccc(F)cc1)[C@@H]1CCOC1)N1CCOCC1', 'O=[N+]([O-])c1ccc2ncn([C@@H](CCl)c3ccccc3)c2c1', 'OC[C@@]1([NH2+]Cc2cccc(Br)c2)CCCC1(C)C', 'O=C(CNS(=O)(=O)c1ccc(Br)cc1)N1CCC[C@@H](c2nc3ccccc3s2)C1', 'O=C(c1cccc(Oc2ncnc(-n3cccc3)c2)c1)N1CCCCCC1', 'COc1nc(NC(=O)Cc2c[nH]c(C)c2C)cc(NCc2ccccc2)c1C', 'O=C(CN(C[C@H]1CCCO1)C1CC1)Nc1cccc(F)c1', 'CC[C@@H](OC(=O)c1cccc([N+](=O)[O-])c1)[C@H]1CCCO1', 'O[C@@H](S[C@@H](Cc1ccccc1)c1ccc(Cl)c(Cl)c1)c1cc(Cl)ccn1', 'Clc1ccc(CN[C@@H]2CCCC[C@H]2[S@@](=O)Cn2cncn2)cc1', 'CN(CCOc1ccccc1)C(=O)CSc1nnc(SCC(N)=O)c(=S)c2ccccc12', 'COCc1noc([C@@H]2CC(=O)c3[nH]nc(C)c3C2)n1', 'Clc1cc(/N=C/C=C/c2ccccc2)nc([O-])n1', 'Clc1nncc(-n2cccc2)c1C1CC1', 'Clc1ccccc1Nc1ncnc2c1CCCC2', 'Clc1ccccc1-c1nc(C2CC2)no1', 'ClC[C@@H]1CCC[NH+](Cc2csc(-c3cccnc3)n2)C1', 'N=C1CCCC1=CCc1c(O)cccc1Cl', 'Clc1cccc(-c2cnsc2)c1', 'O=[N+]([O-])c1cc(C(=O)N[C@H]2CC(=O)N(S(=O)(=O)c3ccccc3)C2)ccc1Cl', 'Clc1ccccc1Nc1cc(S(=O)(=O)N2CCCC2)cc(S(=O)(=O)[O-])c1', 'O[C@@H](S[C@@H](N)c1cccc([N+](=O)[O-])c1)S(=O)(=O)c1ccccc1', 'N[C@@H](Sc1ccc(Cl)s1)C(=O)OCC1CC1', 'Clc1ccc([C@@H](CCN2CCOCC2)[C@@H]2CCS(=O)(=O)C2)s1', 'O=[N+]([O-])c1c(-c2nc3ccccc3o2)ncnc1N1CCOCC1', 'Clc1ccccc1Oc1cnc(N2CCOCC2)c2cc(Br)ccc12', 'ClC[C@H]1OCC[C@@H]1N1CCc2ccccc2C1', 'O=[N+]([O-])c1c(Cl)nsc1N1CC[NH2+]CC1', 'O=C([O-])c1nnc([C@H](c2cccnc2)C2CC2)s1', 'Clc1ccc(-c2cccnc2)n2nnc(-c3ccc([N+](=O)[O-])cc3)c12', 'O=[N+]([O-])c1cc(C2=NN(Cc3ccccc3Cl)NC2=O)cc(Cl)c1Cl', 'Oc1cc(Br)ccc1OCc1ccccc1OC[C@@H]1CCCO1', 'C=C1[C@H](C(=O)Nc2cc(Cl)c(C)cc2C)S[C@@H](C)C[C@H]1C', 'Cc1ncsc1CCN(C)C(=O)CCNC(=O)OC(C)(C)C', 'Clc1cccc(Nc2cnccc2S(=O)(=O)/N=C/c2ccc(F)cc2)c1', 'Clc1nnc(SCC2CC[NH2+]CC2)nn1', 'Clc1ccccc1NS(=O)(=O)c1ccc(Br)c([N+](=O)[O-])c1', 'CN(c1ccc(F)cc1)S(=O)(=O)CC(=O)c1ccccc1', 'Clc1ccccc1N1Nc2ncccc2C1', 'COCc1cccc(NC(=O)C(=O)N2CCC[C@H](C(=O)Nc3ccccc3)C2)c1', 'Cc1sc(N(CC[NH3+])Cc2ccccc2)nc1C', 'O=C(/C=C/c1c(F)cccc1Cl)Oc1ccccc1[N+](=O)[O-]', 'N[C@@H](Sc1ccc(OCc2ccccc2F)cc1)c1ccn(C(F)F)n1', 'Clc1ccccc1Cn1nnc(-c2cccc(F)c2)n1', 'Clc1ccc(-c2n[nH]cc2C2CC2)cc1', 'CCn1nc(C)c([C@@H](NC(=O)Nc2ccc(Cl)c(C#N)c2)c2ccccc2)c1C', 'O=C(/C(c1cccs1)S(=O)(=O)c1ccc(Cl)cc1)c1ccc(F)cc1', 'O=C([C@H](Cl)c1ccccc1F)N1CCN(c2ccccc2)CC1', 'Clc1ccccc1Oc1cc(OCC(F)(F)F)ccc1O', 'N#Cc1nccn1/c1ccc(OCc2ccccc2)cc1', 'Oc1cccc(NC2CCC([NH2+]CCc3c[nH]nc3C)CC2)c1', 'Clc1cccc(-c2nnc(NC[C@@H]3CCCO3)[nH]2)c1', 'Clc1ccc(-c2nnnn2-c2cc(Cl)ccc2Cl)cc1', 'O=[N+]([O-])c1c(OC[C@H]2CCCO2)ccnc1Sc1ncccn1', 'Nc1[nH]ncc1C[S@](=O)Cc1ccccn1', 'O=C(/N=C/c1ccccc1F)N(Cc1ccccc1Cl)c1cccs1', 'Oc1cccc2cc(/C=C/c3ccccc3)ccc12', 'ClCc1ccc(C(=O)N2CCC(Oc3ccc(F)cc3F)CC2)cc1', 'O=C(/C([O-])(c1ccc(F)cc1)C(F)(F)F)c1c(Cl)cccc1Cl', 'Clc1ccc(COc2nc([O-])c(C(=O)[O-])cc2Cl)cc1', 'Clc1cccc([C@@H]2[C@@H]3CCC[C@@]3(CO)C2)c1', 'Clc1ccc(Br)cc1Sc1nc2ccccc2cc1F', 'OC[C@H](SC1=NCCS1)c1ccccc1', 'Clc1ccccc1-c1ncc(Cl)s1', 'Clc1ccccc1-c1csc(Cl)n1', 'Clc1ccccc1-c1csc(Cl)n1', 'Clc1ccc(-c2cccs2)nc1Cl', 'O=C([C@H](Nc1nc2c(Cl)cccc2s1)c1cccs1)Nc1c(F)cccc1F', 'Clc1cc(Br)c2nc(OC[C@H]3CCCO3)ncc2c1O', 'Clc1cc(C[NH2+][C@H]2CCc3c(Br)cccc32)ccc1Cl', 'Cc1ccsc1/C=C/C(=O)N(Cc1ccccc1)Cc1ncns1', 'Clc1ccc(-c2csc3[nH]ccc23)cc1', 'Nc1cncc(/[NH+]=C/c2ccc(C#N)cc2)n1', 'Clc1ccc(-c2cccnc2Cl)cc1Br', 'Clc1cccc(SC[C@@H](O)c2ccc(Cl)cc2F)c1', 'O=C(CN(CC(F)F)c1ccccc1)Nc1ccc2c(c1)C(=O)NCC2', 'Oc1ccccc1OS(=O)(=O)c1ccccc1NC(=O)CC1CCCC1', 'N[C@@H](c1cccc(Br)c1O)[C@@H]1C[NH+]2CCN1CC2', 'Oc1cc(Cn2nccc2C(=O)Sc2nc3ccccc3[nH]2)ccc1F', 'Cc1snnc1S[C@@H](C)C(=O)N[C@@H](c1ccc(F)cc1)C1CC1', 'Nc1cn[nH]c1I', 'Nc1cn[nH]c1Br', 'Clc1cccc(Br)c1S[C@@H]1CN(C(=O)c2cccc(F)c2)CC1', 'COc1nc(-c2cccc(OC)c2)cc(N2CCCC2)c1[N+](=O)[O-]', 'Cc1snnc1CNC(=O)N(C[C@H]1COCCO1)C1CC1', 'Clc1nncn1Cc1ccc2c3c(cccc13)CCC2', 'O=C([O-])/C=C/c1cnn(-c2ccc(Br)cc2)c1N', 'Clc1ccccc1-c1csc(-c2cccc(Br)c2)n1', 'O=C([O-])/C(CNC(=O)c1cc(-c2cccs2)on1)c1ccccc1', 'O=C(c1cnncc1Cl)N1CC=C(c2ccc(F)cc2)CC1', 'Clc1nncnc1Nc1ccc(-c2ccccc2)cc1', 'O=C(Cc1cc(F)ccc1O)NNC[C@@H](O)c1cscc1F', 'O=C(/C=C\\c1nc(-c2ccc(Cl)s2)cs1)c1ccccc1', 'Clc1cccc(F)c1C[S@@](=O)[C@@H]1CC(=O)N(Cc2ccc([N+](=O)[O-])cc2)C1', 'Clc1cc(NSc2ccc(Cl)nc2Cl)ccc1[N+](=O)[O-]', 'Oc1cccc(N2CCN(Cc3ccc(Cl)nc3Cl)CC2)n1', 'O=[N+]([O-])c1ccncc1N1C[C@H](O)CC[C@@H]1c1ccc(Cl)cc1', 'COc1ncnc(Nc2ccc(Cl)cc2[N+](=O)[O-])c1N', 'Clc1cc(/N=C\\C=C/C[NH+]2CCCC2)ccc1Cl', 'Oc1cccc(-c2cn[nH]c2)c1', 'Clc1cc(Cl)nn1/N=C\\c1ccc(Cl)cc1', 'Clc1nncnc1[N-]OCC(=O)c1cccs1', 'Clc1cccc(/N=C2\\O[C@H]3CCCC[C@@H]32)c1F', 'Oc1ccccc1OSCc1noc(-c2ccccc2)n1', 'Clc1ccc(Br)cc1C[C@@H]1CCCN1c1nc(F)ccc1F', 'O=[N+]([O-])c1c([O-])[nH]c(=O)c2ccc(NC(=O)/C=C/N3CCCC3)cc12', 'Clc1cccc2ccc(S[C@H](Cc3ccco3)[NH+]3CCCC3)cc12', 'Clc1ccc(-c2nonc2C2CC2)cc1', 'Clc1ccc(-c2nonc2C2CC2)cc1', 'Clc1ccc(-c2nncn2C2CC2)cc1', 'ClCc1ccnnc1Cl', 'Oc1ccccc1[C@@H]1CCCN1c1nc2ccccc2[nH]1', 'O=[N+]([O-])c1cccc2c1nc([O-])c1ccccc12', 'Clc1cccc2[nH]nc(Sc3nc4ccccc4s3)c12', 'O=[N+]([O-])c1c(Cl)nc2ccccn12', 'Clc1nnn(-c2ccccc2)c(=O)c1Br', 'Clc1ccccc1Oc1cncc2ncnc2c1O', 'Clc1ccccc1Nc1nn[n-]n1', 'Oc1cc(O[C@@H]2O[C@H](CO)c3ccccc32)ccc1OCC(F)(F)F', 'O=C(CNS(=S)(=O)c1ccccc1)NC1CC1', 'O=[N+]([O-])c1c(OCOc2cccc(Cl)c2Cl)cccc1[N+](=O)[O-]', 'C=C[C@@H]1N(C(=O)c2ccccc2)Cc2ccc(OC)cc21', 'Clc1ccc(Br)c(-n2nc(Br)ccc2=O)c1O', 'Clc1cccc2nn[nH]c12', 'Clc1cccc2nn[nH]c12', 'Clc1cccc2[nH]nnc1c1ccccc12', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n', 'Clc1cccc2[nH]nnc12\n\n', 'Clc1cccc2[nH]nnc12\n', 'Clc1cccc2[nH]nnc12', 'Clc1cccc2[nH]nnc12', 'Clc1cccc2[nH]nnc12', 'Clc1cccc2[nH]nnc12', 'Clc1cccc2[nH]nnc12', 'Clc1cccc2[nH]nc(N)c12', 'Clc1cccc2[nH]nc(N)c12', 'Clc1cccc([S@@]CCS(=O)(=O)c2ccc(F)cc2)c1', 'O=C(/C=C/c1ccncc1)N/N=C/c1ccc([N+](=O)[O-])cc1', 'Oc1cccc(CCNc2c([N+](=O)[O-])c(O[C@@H](C)c3ccccc3)no2)c1', 'OCc1n[nH]c(=S)n1/N=C/c1ccc2[nH]cc(-c3ccc(Cl)cc3)c2c1', 'CN[C@H](c1ncc(C(C)(C)C)cn1)c1ccc(Cl)cc1', 'O=[N+]([O-])c1ccn([C@H](c2ccsc2)Cn2cc[nH+]c2)c1', 'O=[N+]([O-])c1cc(Br)c([O-])cc1NCC(F)(F)F', 'ClCc1csc([C@H]2CCCO2)n1', 'O=C(C[NH+]1CC(=O)N2CCC[C@@H]2C1=O)c1ccc(F)cc1', 'Clc1ccccc1OCc1ccc2c(c1)CCC(=O)N2', 'Cc1ncsc1C[NH+]1CC[C@H](Oc2ccccc2)C1', 'Oc1ccc(/C=N/S[C@H]2CCS(=O)(=O)C2)cn1', 'O=[N+]([O-])c1ccncc1O[C@@H]1CCS(=O)(=O)C1', 'Clc1ccccc1NS[C@H](O)c1ccc(Br)cc1', 'O=C(c1c[nH]c2ccccc12)N1CCC[C@H](n2ccnc2)C1', 'Oc1ccccc1[C@H]1N=NC[C@@H]1c1nc(-c2ccc(Cl)cc2)no1', 'OC[C@@H](S)c1cc(Br)ccc1Cl', 'Cc1snc(C[NH+]2CCC(C(=O)NCc3scnc3C)CC2)n1', 'N#CCOc1ccc(/C=N/c2c[nH]cn2)cc1', 'OCS(=O)(=O)c1ccc(N2CCN(c3ccccc3)CC2)cc1', 'ClCc1cccc([C@@H]2CCCN(CN3CCOCC3)C2)c1', 'CN[C@H](Cc1ccc(C)cc1)c1ccc(C)o1', 'Clc1ccsc1N[C@@H]1CCOC2(CCOCC2)C1', 'O=[N+]([O-])c1c(Br)cc(I)cc1O[C@@H]1C=CS(=O)(=O)C1', 'OCS1(=O)CCN(C(=O)c2ccc(F)cc2F)CC1', 'Clc1nncc(NC[C@@H]2COc3c(F)cccc32)n1', 'Clc1ccc([C@@H](O)[C@@H](Cc2cccc(F)c2)c2cccnc2)cc1', 'Clc1ccccc1OC1C[NH2+]CC1', 'Clc1ccccc1OC1CC[NH2+]CC1', 'Oc1ccc([C@H]2CCOCC2)cc1', 'Oc1cccc(-c2cn(-c3ccc(Cl)cc3Cl)[nH]c2=O)c1', 'Clc1ccc(C2C(C(=O)NCc3cccc(C(F)(F)F)c3)CC2)cc1', 'O=C(/N=C1\\CC[NH+]=C1C1OCCO1)c1cnc(Cl)cc1Br', 'Clc1cc(F)cc(N[C@H]2CCN(c3ccccc3)C2)c1', 'C[C@@H](c1ncoc1C1CC1)c1nnc(NC(=O)c2cccc(Cl)c2)s1', 'Clc1ccc(Cc2csnn2)cc1NCCO', 'Nc1n[nH]c([C@@H]2CCCCC[C@@H]2[NH3+])n1', 'O=C([O-])[C@H]1[C@H]2C=C[C@@]2(O)CCN1c1cccnc1', 'Clc1cc(Cl)ncc1F', 'Clc1cc(Cl)ncc1Cl', 'Clc1cc(Cl)ncc1Br', 'Clc1cc(Cl)cnc1Cl', 'O=C(/C(CO)c1ccc(Cl)cc1)c1ccc(Br)c([N+](=O)[O-])c1', 'Clc1cccc(-c2cnc(C[NH+]3CCN(Cc4ccc(F)cc4F)C3)o2)c1', 'CCc1ccc(NC(=O)N2CCN(c3cccc(C)c3)CC2)cc1[N+](=O)[O-]', 'Clc1ccccc1NS(=O)(=O)c1cc2c(cc1Cl)OCCO2', 'Clc1ccc(Br)cc1S(=O)(=O)N[C@@H]1CCSc2ccccc21', 'O=C(CSC(=O)Nc1ccc2c(c1)nc1ccccn12)N1CCC(Cc2ccccc2)CC1', 'O=C(c1cccc(Cl)c1)N1CCC[C@]2(CCN(S(=O)(=O)c3ccccc3)C2)C1', 'O=C(CSCNSc1ccccc1)c1ccccc1F', 'Clc1ccc(-c2cccs2)nc1SCc1ccccc1Cl', 'NS(=O)(=O)/N=C/c1ccc(-c2ccc(Cl)cc2)cc1', 'Oc1ccc([C@H](CN2CCCCCC2)c2ccccc2)c(F)c1', 'OCc1nn(Cc2ccco2)c2[nH]ccc12', 'CNC(=O)c1csc(C(=O)O[C@@H]2CC[C@H](Sc3ccccc3)C2)c1', 'Clc1cc(NSc2ncnc3ccccc23)c2ccccc2n1', 'Clc1ccc(CO)c(N2CCN(Cc3ccco3)CC2)c1', 'O=C(c1ccsc1)[C@@H]1Cc2ccccc2C(=O)O1', 'C[C@@H](OC(=O)[C@@H]1CCCO[C@H]1C)c1ccc2c(c1)OCCCO2', 'Clc1cccc2c(N3CCOCC3)nc3ccccc3c12', 'O=[N+]([O-])c1ccc(C[NH+]2CCC(c3nc4ccccc4o3)CC2)cc1', 'COc1ncncc1-c1ccc(Br)c(S(=O)(=O)N2CCOCC2)c1', 'Clc1cccc(F)c1O[C@H]1C[C@H]2CC[C@@H]1C2', 'Clc1ccccc1Oc1cnc(N[C@@H]2CCC[C@@H](C(F)(F)F)C2)[nH]1', 'Oc1ccc(F)c(Cl)c1Cn1ccnc1', 'Clc1nnnc(Cl)c1Cc1ccc(Cl)cc1', 'Clc1nncn1-n1cccc1', 'Clc1ccccc1-c1nc(CCl)on1', 'Clc1ccsc1/N=C\\C(=O)NCCN1C(=O)/C(=C/c2ccc(Cl)cc2)SC1=S', 'N[C@@H](Sc1nnc2ccccn12)[C@@H]1CCCO1', 'Oc1cccc(SC2[NH+]=c3ccccc3=[NH+]2)n1', 'O=C(c1c[nH]cn1)N1CCCSCC1', 'ClCc1cn[nH]c1N1CCCSCC1', 'Clc1ccccc1Oc1cnc(N2CCCCC2)cc1Cl', 'N#C[C@@H](c1ccccc1)[C@H]1CCCN1C(=O)N[C@H](c1cccc(C)c1)c1ccccc1', 'O=C(CSCc1cccnc1)NN[C@@H]1CCc2ccccc21', 'ClCc1cn[nH]c1[C@@H]1CCCN1c1ccc(F)cc1C(F)(F)F', 'Clc1ccc(Br)cc1-n1nnc(-c2cccs2)n1', 'Clc1ccccc1CSc1cncc(OCC(F)F)n1', 'NS(=O)(=O)/C(CNC(=O)c1ccc(Cl)cc1)c1ccc(C(=O)[O-])cc1', 'Clc1ccc(COc2nc3cccc(Cl)c3[n-]2)cc1', 'Oc1cccc2cc(COc3cccc(C[N+](=O)[O-])c3)[nH]c12', 'O=C(CCOc1ccc(Cl)cc1)N(Cc1cccc(F)c1)C(=O)c1nc2ccccc2c(=O)[nH]1', 'O=C(/N=C\\NCCc1cccc(Cl)c1)c1nc([O-])c2ccc(Cl)cn12', 'Clc1cccc2nc([C@@H](CO)NNC(=O)c3cccc(Cl)c3)nc([O-])c12', 'Oc1ccc(O[C@@H]2C[C@@H]3CCCC[C@H]3C2)cc1OCC(F)(F)F', 'O=[N+]([O-])c1c(Cl)n2nnc(-c3ccccc3)c2cc1O', 'O=C(Cc1n[nH]c2c(Cl)cccc12)Nc1ccccc1F', 'Clc1cc(CNC[C@](O)(c2ccccc2)c2ccc(C#N)cc2)cc([O-])n1', 'Oc1ccc(/N=c2/[nH]cc[nH]2)nc1', 'Clc1ccc(-c2csc3sc4c(c23)CCCC4)cc1', 'Oc1ccc([N+](=O)[O-])c(Cl)c1Oc1cccc2ccccc12', 'O=C(c1cc(Cl)ccn1)N1CCOCC1', 'Clc1cc(Cl)nnc1N1CCCCC1', 'Clc1cc(Cl)nnc1N1CCCCC1', 'O=C(CNC(=O)CN1C(=O)[C@@H]2CC=CC[C@H]2C1)Cc1cc2ncccc2s1', 'Clc1cc(C2CC[NH+](C[C@H](c3cccnc3)c3cccs3)CC2)ncn1', 'Clc1cc(/C=N/Nc2nc(-c3ccccc3)n[nH]c2=O)cs1', 'COc1ncc(C)cc1OC(=O)[C@@H]1Cc2ccccc2CN1C(=O)C(C)C', 'Clc1cc(Nc2nc3ccccc3[nH]c2=O)cnc1Sc1ccc(F)cc1Cl', 'Clc1ccc(Cc2noc(-c3ccco3)c2Cl)cc1', 'Oc1ccccc1Oc1cncc([O-])n1', 'O=[N+]([O-])c1c([O-])cnn1-c1ccccc1', 'Clc1cccc2ncn([C@@H](CO)C(=O)N3CCCC3)c12', 'Clc1ccc(C[NH+]2CCC[C@H](c3nc(-c4cccnc4)no3)C2)s1', 'Oc1ccc([N+](=O)[O-])c(Cl)c1Oc1ncc(Cl)c(Cl)c1Cl', 'Oc1cccc(CCn2ccnc2NC[C@@H](O)C[NH+]2CCCC2)c1', 'Clc1ccc(-c2n[nH]c3c2[nH]c(=S)n3C(F)(F)F)cc1', 'Clc1ccccc1O/N=C/Nc1c(Cl)cccc1[N+](=O)[O-]', 'Clc1ccccc1Nc1nnnn1-c1ccccc1F', 'Oc1cc(CCNCc2nc3cc(Cl)ccc3s2)ccc1F', 'Clc1cc(NSc2nncn2C2CC2)ccc1Oc1ccccc1', 'Oc1cc(Cn2cccc2)ccn1', 'Oc1cccc([C@H]2CNc3ccccc32)c1F', 'Clc1ccccc1Oc1cc(SC(F)(F)F)ccc1Cl', 'Clc1ccccc1Oc1cc(S(=O)(=O)N2CCCCC2)ccc1Cl', 'Clc1ccccc1Oc1cc(S(=O)(=O)N2CCCC2)ccc1Cl', 'O=C(c1cc(Cl)c[nH]1)N1CCc2ccc(NN)cc21', 'ClCc1n[nH]c(O)c1C(F)(F)F', 'Oc1ccc(F)cc1[C@H]1[C@@H]2C(=O)N(c3cccnc3)[C@@H]2[C@@H]2C=CC[C@H]21', 'O=C([O-])[C@@H]1C[C@@H]2CC[P@]1[C@H]1O[C@@H]1[C@@H](c1ccc(Cl)cc1)[C@@H]1C[C@H]1O2', 'ClCc1csc(C2CC2)n1', 'ClCc1csc(C2CC2)n1', 'Oc1ccc([N+](=O)COCc2ccco2)cc1Cl', 'O=C(c1cccs1)c1ccnc(OC2CCCCC2)c1', 'Oc1ccc([C@H](Nc2cc(Cl)ccc2F)N2CCCCC2)s1', 'Clc1ccc(Br)cc1-c1cn(-c2ccccn2)nc1C1CC1', 'OCS[C@@H]1CCCc2[nH]nc(-c3ccccc3)c(=O)c21', 'Oc1ccc([N+](=O)CNc2ccccc2OCC2CCCCC2)cc1', 'O=[N+]([O-])c1c(-c2ccco2)sc2ccccc12', 'O=C([O-])/C(C#N)[C@@H](c1ccccc1Cl)c1c[nH]c2ccccc12', 'O=C(CS(=O)(=O)c1ccc(Cl)c(Cl)c1)Nc1ccc(F)cc1', 'O=[N+]([O-])c1c(OCO)cccc1N1CCCC1', 'Oc1ccc(Oc2cn(-c3ccccc3)ccc2=O)nc1O', 'ClCc1ccccc1O/N=C1/N=C(c2ccccc2)SC1=O', 'Clc1ccccc1O/C=C1/SC(=S)N(CC(=O)Nc2ccccc2)C1', 'Oc1ccc([C@H](NC(=O)c2cccc(Cl)c2)c2ccccc2)cc1', 'N=CCc1ccc(Cl)c(CCC(=O)Nc2ccc(F)cc2)c1', 'Clc1ccc([C@@H]([O-])C[C@@H](/C=O)c2cc(F)ccc2F)cc1', 'N=Cc1cnc(C2CC2)nc1-c1cccc([N+](=O)[O-])c1', 'Clc1ccc(Br)cc1[C@@H]1C[C@@H](CI)c2[nH]cnc2S1', 'Clc1ccc([C@@H]([C@@H]2Cc3cnccc3S2)N2CCOCC2)cc1', 'Clc1ccccc1NC(=O)CSc1nnnn1-c1ccccc1', 'Clc1ccc(Br)cc1Sc1cccc(F)c1', 'Oc1cccc(NCc2ncc(-c3ccccc3)o2)c1O', 'Oc1ccccc1/N=N\\Nc1cc([N+](=O)[O-])ccc1C[NH2+]C1CC1', 'Clc1ccc(Cc2ncc[nH]2)cc1F', 'Oc1cc(OCOCCOc2ccc(C(F)(F)F)cc2)ccc1Br', 'O=C([O-])[C@H]1[C@H]2C=C[C@@H]1[C@@H](c1ccc(OC(F)F)cc1)N2', 'ClCc1ccccc1OC(F)F', 'O=C(c1cc([S@](=O)Cc2cccs2)nc2ccccc12)c1ccc(F)cc1', 'ClCc1csc(N2[C@H](C(=O)[O-])CCOC2)n1', 'O[C@@H](c1csnn1)[C@H]1C[NH+]2CC[NH+]1CCC2', 'O[C@@H]([C@@H]1CCOc2ccccc21)[C@@H]1CCOC2(CCOCC2)C1', 'Oc1cc(C(=O)N2CC=C(CCn3ccnn3)CC2)ccc1O', 'Clc1cc(NCCc2ccccc2Cl)ncn1', 'Clc1ccc(-c2nnnn2CC2CC2)cc1F', 'Oc1cccc(Br)c1OCBr', 'C[C@@H](c1nnc(S[C@@H](C)C(=O)N2CCCc3ccccc32)s1)n1cccn1', 'Clc1ccccc1NC1CSC1', 'ClCc1csc(COc2ccc(Br)cc2)n1', 'Clc1ccsc1C(F)(F)Cc1ccccc1Cl', 'Oc1ccccc1[C@H]1Oc2ccccc2Oc2ccccc21', 'Cc1ncsc1C(=O)N1CCC[C@@H](Oc2nc3ccccc3cc2CO)C1', 'Clc1ccc(-c2nnnn2-c2ccc(F)cc2F)cc1', 'Clc1ccc(-c2nnnn2-c2cc(F)ccc2F)cc1', 'O=[N+]([O-])c1cc(Br)cc(SCc2ccccc2)c1', 'Clc1cccc([S@@]c2cnccc2[O-])c1', 'Clc1ccccc1[C@H](Br)C1CCC(O)CC1', 'Clc1ccc(Br)cc1OS(=O)(=O)Cc1ccccc1', 'Clc1ccccc1OCC1NCCN(C(=O)Cc2ccc(C#N)cc2)CC1', 'Clc1nnnn1/C(c1cccs1)[C@H]1C[C@H]1c1cccc(Cl)c1Cl', 'Oc1cc(OCOc2cc(-c3ccccc3)n[nH]c2=O)cs1', 'Clc1ccc(Br)cc1-c1cnn(CCN2CCCC2)c1', 'Clc1ccccc1CN(C(=O)/C=C/c1ccc(Cl)cc1Cl)c1ccccc1', 'O=C([O-])[C@@H]1CN([C@@H]2CCCc3c(F)cccc32)C1', 'Nc1[nH]nc2c1-c1nc(N)ccc1N2', 'O=[N+]([O-])c1cc(SC[C@@H]2Cc3ccccc3O2)c[nH]c1=O', 'O=C(CSCNOc1cccc(C(F)(F)F)c1)c1cccc([N+](=O)[O-])c1', 'Clc1cccc(NC2CCCC2)c1', 'NS(=O)(=O)/C(=C\\c1ccco1)CN1C(=O)CN(CCO)C1=O', 'N#CC1(SCc2ccco2)CCCCC1', 'Oc1ccc([N+](=O)c2onc3c2CCCC3)c2ccccc12', 'Oc1ccc(O[C@H]2CCOC2)cc1-n1cccn1', 'O=C(c1c[nH]c2ccc(Br)cc12)N1CCSCC1', 'Oc1cc(C=NC(=O)c2cccc(Br)c2)ccc1OC(F)F', 'Nc1nonc1C[NH+]1CC[C@@H](OS(=O)(=O)N(CCCCO)c2cccs2)C1', 'Nc1[nH]nc(Sc2ccccc2)n1', 'O[C@@H](Sc1nnc(-c2ccc(F)cc2)o1)c1cc(Cl)ccc1Cl', 'Clc1cc(/C=N\\OC(=O)COc2ccccc2Cl)c2c(c1)OCO2', 'O=[N+]([O-])c1cc(N[C@@H](CO)c2ccc(F)c(F)c2)cn1C1CCCC1', 'Oc1cccc(-c2cc3occc3nc2NC2CCC(CO)CC2)c1', 'Clc1ccccc1Cc1cccc(OC2CC[NH2+]CC2)c1', 'OC1=C2CCCC[C@@H]2C(=O)N1C(=O)c1ccc(Br)s1', 'Clc1cc(NSc2cccs2)ccc1OS(=O)(=O)c1ccccc1', 'Clc1cccc2cc(Br)[nH]c(=O)c12', 'Clc1ccc(Br)cc1Sc1nc2ccccc2n1Cc1ccccc1Cl', 'Clc1ccc(Cc2noc(-c3ccc([N+](=O)[O-])cc3)n2)cc1', 'CN(CC[NH+](C)C)C(=O)c1ccc2c(c1)N(C)c1ncccc1N=C1C=CC=C1C=C2', 'Clc1ccc(-c2cc(N3CCOCC3)[nH]c2Br)nc1', 'Oc1cc(-n2cc[nH+]c2C2CC2)cc(N2CC(=O)Nc3ccc([N+](=O)[O-])cc32)c1', 'Oc1cccc(N2CC[NH2+]CC2)c1O', 'Clc1ccc(Cc2ccc(OCc3ccc(Cl)cc3Cl)o2)cc1', 'Clc1ccc(Br)cc1-c1cn(-c2c(F)cccc2F)nc1N1CCCCC1', 'ClC([O-])/C=C/c1ccc2nc(-c3cccc(F)c3)[nH]c2c1', 'Clc1ccc(C(F)(F)F)c(-[n+]2c(C(F)(F)F)nc3ccccn32)c1', 'Clc1ccccc1-c1nc2ccc(Cl)cc2s1', 'Clc1ccc(-c2csc3[nH]cnc32)c(C(=O)Nc2ccc(Br)cc2)c1', 'O=C(CNCc1nc2ccccc2[nH]1)Nc1cc(C(=O)[O-])cc(Cl)c1', 'Oc1cccc2cc(CC[NH2+]Cc3ccncc3)ncc12', 'C[C@@H](NCc1ccc(Cl)cc1Cl)c1ccc(Cl)cc1', 'C=C1N=C(C(=O)Nc2nncs2)[NH+](Cc2ccc(OC)cc2)C1=O', 'Oc1ccccc1/N=N\\S(=O)(=O)CCNC(=O)Nc1ccc(I)cc1', 'Clc1ccccc1-c1noc(CCCS(=O)(=O)c2ccccc2)n1', 'O=C(c1c[nH]c2cc([N+](=O)[O-])ccc12)[C@@H](Cc1ccccc1)n1cccn1', 'Nc1[nH]ncc1N1CCN(C(=O)c2c(Cl)cccc2Cl)CC1', 'O=C(/N=C/NC(=S)N1CCCC1)Sc1cccc(I)c1', 'Clc1ccccc1O/Cn1c2ccccc2[nH+]2ncnc21', 'O=C(c1ccsc1)[C@@H]1CCCN(Cc2ccccc2F)C1', 'O=C([C@H]1C[C@H]1c1cccs1)N1CCN(c2ccc(F)c(F)c2)CC1', 'COc1nc([O-])cc(C(=O)NCC[C@H](O)c2ccccc2C)c1N', 'Clc1nnnn1-c1ccc(O)cc1', 'Nc1[nH]nc2ncn(Cc3ccccc3Cl)c12', 'COc1nn[nH]c1C(=O)Nc1ccccc1S(=O)(=O)c1ccc(F)cc1', 'Clc1nnnn1-c1c(Br)ccc(Br)c1[O-]', 'Clc1cccc2c(N)[nH]nc12', 'O[C@@H](Sc1nnc(-c2ccccc2)o1)C(=O)Nc1ccccc1', 'Clc1ccccc1-c1nc(CS(=O)(=O)N2CCCCC2)c[nH]1', 'Clc1cc(N2CCC[C@@H](N3CC[NH+](CCO)C3)CC2)c[nH]1', 'Clc1cccc([S@@]CC2(Cn3cc(Cl)cn3)CC2)c1', 'Clc1ccc(C2SCCS2)cc1O', 'Clc1cccc2cc(CNC[C@@H]3COc4ccccc4O3)nn12', 'Clc1cc(/N=C/C2CCOCC2)ccc1OCCCO', 'O=C(/C=C/c1cccnc1)Nc1ccc([N+](=O)[O-])c(Br)c1', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n', 'Oc1cccc(F)n1\n\n\n', 'Oc1cccc(F)n1\n\n', 'Oc1cccc(F)n1\n', 'Oc1cccc(F)n1', 'Oc1cccc(F)n1', 'Oc1cccc(F)n1', 'ClCc1ncn[nH]1', 'ClCc1nc[nH]n1', 'ClCc1cnn[nH]1', 'Oc1cccc(N2CCCC2)c1', 'Clc1cc(NSc2nnnn2C2CCCC2)ccn1', 'Oc1ccc([N+](=O)c2ocnc2)c2[nH]ccc12', 'Oc1ccc(Oc2ncn[nH]2)cc1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1O', 'O=[N+]([O-])c1ccc2[nH]ncc2c1O', 'Nc1cnn(-c2ncccn2)c1Br', 'Nc1[nH]nc(-c2ccco2)c1N', 'Clc1cc(F)cc2nc(CN3CCCCCC3)nn12', 'Cc1sc(C#N)c(S[C@@H](C(=O)[O-])c2ccc(Cl)cc2)n1', 'O=C(c1ccco1)[C@@H]1CCCN1C(=O)CSc1ccc(F)cc1', 'Clc1ccccc1C[NH+]1C[C@@H]2CCC[C@@H](O)[C@@H]2C1', 'O=[N+]([O-])c1c([N+](=O)[O-])c(Br)ccc1OC(F)F', 'CN(C)C(=O)c1ccc(NC(=O)c2ccc([N+](=O)[O-])o2)cc1', 'Clc1cc(Br)cc(CCOCc2ccc(F)cc2)c1', 'Clc1ccc(-c2ccccc2C(F)(F)F)cc1', 'Nc1nonc1C(=O)N1CCC(c2ncc(C(C)(C)C)nc2)CC1', 'Clc1cccc2c(Oc1[O-])nc(Cn1cccn1)nn2C1CC1', 'Clc1cccc(NCCn2cc([N+](=O)[O-])cn2)c1', 'Clc1ccccc1Oc1cnc(C(C)C)cc1O', 'Clc1cc(F)c([N+](=O)[O-])cc1OCc1ccccc1[N+](=O)[O-]', 'O=C(CCS)NCc1ccc(C(F)(F)F)cn1', 'Clc1cc(F)c(C[NH+]2CCCC[C@@H]2CO)cc1F', 'Oc1cc(O)c2scc(Br)c2c1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn2c(=O)n(-c3ccccc3)nc12', 'Oc1cc(F)cc(C[NH+]2CCC[C@H]2c2cnoc2)c1', 'O=C(/N=C/c1ccco1)c1ccnc(N2CCCCC2)c1', 'O=C(Cc1noc2c1CCCC2)Nc1ccc(Br)cc1', 'Oc1ccc(Oc2cnc(C[NH+]3CCCCC3)s2)cc1', 'C[C@@H](c1ccsc1)N(C)C(=O)c1coc(C2CC2)n1', 'O=C(c1c[nH]c2cc(C(F)(F)F)cnc12)N1CCC(c2cnc[nH]2)CC1', 'ClC[C@H](c1ccco1)n1cncn1', 'Clc1cc(F)c([C@H](Cn2cc(Br)cn2)N(C2CCCC2)S(C)(=O)=O)cc1', 'C=C[C@@H](Sc1cccc(F)c1)C(=O)N/N=C/c1ccc(C)cc1', 'O=C(c1cc(Cl)cnc1N1CCCCC1)N1CCN(C(=O)c2cccnc2)CC1', 'Clc1cc(Br)c2nc(Oc3ccccc3F)[nH]c(=O)c2c1', 'ClCc1n[nH]c2cc(-c3cc(F)cc(F)c3F)oc12', 'Clc1ccc(Br)cc1-c1cnn(-c2ccc(Cl)cc2Cl)c1', 'Oc1cccc(OCCc2cccnc2)c1O', 'Clc1cccc2c1[C@@H](Cn1ccnc1-c1ccc[nH]c1=O)C2', 'Oc1ccc([N+](=O)Cc2noc3ccccc23)c2ncccc12', 'N=C1C[C@]2(CCC[NH+](Cc3ccccc3)n2)Nc2cc(Cl)ccc2N1', 'Clc1ccccc1Oc1cnc2cc(S(=O)(=O)N3CCN(c4cccc(Br)c4)CC3)ncc2n1', 'Cc1snnc1S(=O)(=O)c1ccc(S(=O)(=O)N(C)C)cc1', 'Oc1ccccc1[C@H]1OCC[C@@H]1F', 'N[C@@H]1c2ccccc2C[NH+]1C/C([O-])F', 'Oc1cccc2cc(/C=C(\\Cl)c3ccccc3)c(=O)c(=O)[nH]c12', 'O=[N+]([O-])c1c(-c2cc([N+](=O)[O-])ccc2)cc[nH]1', 'O=C(c1ccc(OS(=O)(=O)c2ccco2)cc1)[S@@](=O)c1ccc(Cl)cc1', 'Clc1ccccc1-c1csc(Sc2cn[nH]c2-c2cc(Cl)ccc2F)n1', 'Clc1ccccc1N1N=CC(c2ccccc2)c2ccccc21', 'Clc1ccc(C(CO)OC(F)F)cc1', 'O=C(c1cnccc1Cl)N1CCSc2cc(Cl)ccc21', 'O=[N+]([O-])c1ccnc2cc(C(F)(F)F)cc(-c3cc(Cl)cnc3C3CC3)c12', 'N#C[C@@H]1Cc2ccccc2N1C(=O)c1cccc(NC(=O)c2sc(C3CC3)nn2)c1', 'CC(C)CS(=O)(=O)N(C)Cc1ccc(Br)o1', 'Clc1cc(Br)c(CN2CCN(c3cnc4cc(Cl)ccc4c3)CC2)cc1', 'Clc1cc(N2CCOc3ccccc32)nnc1-c1ccccc1', 'Clc1cccc(Br)c1S', 'Clc1cccc([S@@]c2cccc(Br)c2)c1', 'Oc1cccc(Br)c1COc1cccc(Br)c1', 'CNc1nnc(SCC(=O)Nc2cccc(Br)c2)nc1N', 'O=C(c1cc(Br)c[nH]1)N1CCC[C@H](CO)C1', 'Oc1cc(F)c(/C=N/NS(=O)(=O)c2ccccc2)cc1Cl', 'Clc1ccccc1Oc1cc(NCc2cccnc2)ncn1', 'Clc1ccsc1[C@@H]([NH2+][C@@H]1CCCc2cccnc21)C1CC1', 'ClCc1cc(Br)ccc1Oc1ccccc1F', 'O=[N+]([O-])c1cc([C@H]2S[C@@H](O)C[C@H]3CC[C@H]2O3)cc(Br)c1O', 'O=C(COc1cc([N+](=O)[O-])ccc1[N+](=O)[O-])N1CCCC1', 'O[C@@H]1[C@@H](C2CC([O-])O[C@@H]2c2ccc(Cl)cc2)CC1', 'Clc1ccc(-c2csc3sc4c(c23)CCC4)cc1', 'Clc1ccc(-c2nnnn2Cl)cc1O', 'Clc1ccc(-c2nnnn2Cl)cc1F', 'Clc1ccc(-c2nnnn2Cl)cc1Cl', 'COc1nn[nH]c1C(=O)NC[C@@H](C)c1cccc(C)c1', 'Oc1ccc(/C=N\\Cc2ccccc2)c(Br)c1O', 'O=[N+]([O-])c1c(Cl)ncnc1CS[C@@H]1CCc2ccccc21', 'C[C@@H](N[C@@H](CC#N)c1ccccc1)c1ccccc1Br', 'ClCc1cn[nH]c1n1cccn1', 'Oc1ccccc1/N=C/c1cccc(OCC(F)F)c1O', 'Clc1ccccc1Cn1cc([N+](=O)[O-])cn1', 'Clc1ccc(Br)cc1Sc1nc2ccc(F)cc2n1C1CC1', 'Clc1nncn1[C@@H](CO)c1ncccc1F', 'O=[N+]([O-])c1cc(Cl)c(NCc2ccncc2)cc1F', 'O=C(c1c[nH]cn1)OCc1ccc([N+](=O)[O-])c(Cl)c1', 'Clc1ccc(C(N)=O)cc1OCc1nnn[n-]1', 'Clc1ccccc1Oc1cc(F)c(-c2ncoc2C2CC2)nc1', 'O=C(CNS(=O)(=O)c1ccccc1)NNC(=O)c1cc(-c2ccccc2)on1', 'ClCc1csc2c1COCCCO2', 'O=C(C1=NO[C@@H](COc2ccc([N+](=O)[O-])cc2)N1)Nc1ccc(N2C(=O)CCCC2=O)cc1', 'Clc1cccc(/C=N/c2ccc[n+]c2)c1', 'NS(=O)(=O)/N=C(\\C=C1CCCC1)Nc1ccc2[nH]c(=O)[nH]c(=O)c2c1', 'COc1ncc([N+](=O)[O-])c(/C=N/NC(=O)c2ccc(C)cc2OCC)c1C', 'Cc1ccsc1NC(=O)C1CCN(C(=O)[C@@H]2CCO[C@@H]2C)CC1', 'O=C([O-])Cc1ccc(NS(=O)(=O)c2ccccc2F)cc1', 'Nc1c(F)cc(Cl)cc1CN1CC[C@@](Cc2ccccc2)C1', 'O=C(COc1ccc2c(c1)OCOC2)Nc1cccc(F)c1Cl', 'Clc1ccccc1C(=O)O[C@H]1CCOc2ccccc21', 'ClC[C@@H]1OCCCN1C[C@@H](O)c1ccc2c(c1)CCCC2', 'Clc1cc(Cl)nc2[nH]nc(C(=O)C[S@](=O)CC3CC3)c12', 'O=C(c1cc([S@](=O)C23CC4CC(CC(C4)C2)C3)oc1-n1cnnn1)c1ccccc1', 'Clc1ccccc1Cc1ncsc1Br', 'Clc1ccccc1-c1csc(CCl)n1', 'O=C(c1cnnn1-c1ccc(F)cc1)N1[C@@H](C(=O)[O-])C[C@]2(CCCc3ccccc32)N1Cl', 'O=C([O-])/C=C/c1cn(-c2ccccc2)nn1', 'Oc1cc(O)ccc1OCCS(=O)(=O)N[C@@H]1CCc2ccccc21', 'O=C(Cc1cc(S(=O)(=O)N2CCOCC2)ccc1O)c1ccccc1', 'OCc1n[nH]c(COc2ccccc2OCc2cccc(Cl)c2)n1', 'ClCc1csc(OCc2ccccc2)n1', 'ClCc1csc(OCc2ccccc2)n1', 'O=[N+]([O-])c1c(Cl)nc([C@@H]2CCCC[C@@H]2O)nc1-c1ccccc1', 'ClCc1cncc2[nH]nnc12', 'Clc1ccccc1N1Nc2ccccc2C(=O)N1Cc1cccc(F)c1', 'Oc1ccccc1/N=C(\\Nc1c(F)cc(Cl)cc1)c1ccc(Cl)cc1', 'O=[N+]([O-])c1cccc2c1[nH]c(=S)n2CCS(C)(=O)=O', 'Clc1ccc(-c2cc([C@H](C#N)C3CC3)cs2)cc1', 'Clc1cccc(F)c1Oc1ccc(Br)cc1', 'O=C([O-])[C@H](c1ccccc1)n1cnnn1', 'Cc1cnc2c(NCc3ccccc3)nnn2c1', 'O=[N+]([O-])c1c(-c2nc(C3CC3)no2)cnn1Cc1ccc(F)cc1', 'Clc1cccc2cc(O)[nH]c12', 'Oc1ccc(/C=N/Oc2cccc(Br)c2)cc1', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n', 'Clc1cccc2[nH]ncc12\n\n', 'Clc1cccc2[nH]ncc12\n', 'Clc1cccc2[nH]ncc12', 'Clc1cccc2[nH]ncc12', 'Clc1cccc2[nH]ncc12', 'Clc1cccc2[nH]ncc12', 'Oc1cc(COc2ccccc2C=O)c2ccccc2c1O', 'C[NH+](C)Cc1ccc2ccccc2c1Cl', 'O=C(/C=C/O)c1ccccc1', 'NS(=O)(=O)/C=C/CSc1ccc(F)cc1', 'O=C(c1cnccc1)[C@H]1C[C@@H](O)CO1', 'ClCc1cnccc1CCc1nc2c(c(=O)[nH]1)S2', 'O=C(c1cccs1)c1ccc(N2CCN(c3cccc(Cl)c3)CC2)cc1', 'O=C(C[NH+]1CCOCC1)Nc1cccc(Cl)c1', 'ClC[C@@H]1Oc2ccccc2Oc2ccccc21', 'CNC(=O)NNc1ccc(C(=O)NC(C)C)c(Cl)c1', 'COC(=O)c1ccc(Cl)c([O-])c1C(=O)Nc1ccc(OC)cc1', 'Oc1cccc(SC2CCC([NH2+]Cc3cccs3)CC2)c1', 'Clc1nnnc(C2=NC(=O)Nc3ccccc32)n1', 'Clc1ccc(-c2cc(Cl)cn2CCO)cc1', 'Clc1ccc(-c2n[nH]c3[nH]cnc23)cc1F', 'Clc1ccc(-c2n[nH]c3[nH]cnc23)cc1Cl', 'Clc1cc(Cl)nc2[nH]nc(-c3ccccc3)c12', 'N[C@@H](CCOc1cccc(C(F)(F)F)c1)N(C)Cc1ccc(-n2cncn2)cc1', 'C[C@@H]([C@@H](O)c1ccccc1)N1CCc2ncc(C(F)(F)F)cc2C1', 'Oc1ccccc1-n1nnnc1SCCCCC(=O)NCCc1ccccc1', 'OCc1n[nH]c(Sc2nc(-c3ccccc3)no2)c1[N+](=O)[O-]', 'Oc1ccc(F)c(-c2cc3[nH]c(=O)n(-c4ccccc4[N+](=O)[O-])c3s2)c1', 'Clc1cccc(/N=C\\c2ccccc2)c1', 'Clc1cccc(/N=C/c2ccccc2)c1', 'Clc1cc(Cl)c2c3c(ccc2n1)C[C@](O)(Cc1ccc(F)cc1)N3', 'Clc1cccc(NCc2ccc[n+]([O-])c2)c1', 'Clc1ccccc1Nc1nnc2c(Br)ncncc12', 'Clc1ccc(CO)cc1OCc1cscn1', 'Clc1cc(Br)c2ncccc2c1OC[C@H]1CCCO1', 'O=C([O-])[C@@H]1CC(c2ccccc2)=Nc2ncnn21', 'Oc1ccc(/C=N\\C[C@@H]2CSc3ccccc32)cc1', 'Oc1cc(F)ncc1-c1cccnc1NCc1ccccc1F', 'Oc1ccc(/C=C/C(=O)N2CC=C(c3ccccc3)OC2=O)cc1Cl', 'O=[N+]([O-])c1c([N+](=O)[O-])c(F)cc(C(=O)NCCc2ccccc2)c1', 'C[C@@H](Oc1ccccc1)C(=O)Nc1ccc([N+](=O)[O-])c([N+](=O)[O-])c1', 'COc1nccc([C@@H]2N=NC(=O)c3ccc(OC(C)C)cc32)c1C', 'Clc1cccc2[nH]nc(NC[C@@H](c3ccsc3)c3ccccc3)c12', 'ClC[C@H]([NH2+][C@@H](C#N)c1ccccc1)c1csc(Cl)c1', 'Clc1cc(Br)c2nc(OC3CCCC3)oc2c1', 'Oc1cc(F)ncc1C[NH+](Cc1ccco1)Cc1ccco1', 'O=[N+]([O-])c1cc(N2CC(C(=O)NCc3cccs3)C2)ccc1Br', 'O=C(/N=C/c1cccnc1)c1nc2cc(Cl)ccc2o1', 'Clc1ccsc1S(=O)(=O)[C@@H]1CCCO1', 'O=C(c1cccn2c(=O)cc(C3CC3)nc12)N1CCCC1', 'Cc1ccsc1S(=O)(=O)N[C@@H](C)CS(C)(=O)=O', 'Nc1[nH]nc(S[C@H](C(=O)[O-])c2ccccc2)c1-c1ccccc1', 'N=Cc1c(OCCSc2ccc(F)cc2)nc2ccccn12', 'O[C@@H]1CCCCC2(CCN([NH2+]Cc3ccc(OC(F)F)c(F)c3)CC2)C1', 'O=C(Cc1n[nH]c2ccccc12)Nc1ccc2c(c1)CCO2', 'Clc1ccc(-c2nonc2N2CCc3ccccc3CC2)cc1', 'Oc1ccc(/C=C(/C#N)c2cnc(Cl)cn2)cc1', 'ClCc1cn[nH]c1', 'OC1=CN=C2S[C@@H](COC3=c4ccccc4=[NH+][C@H]3C(=O)N2C2CC1)SCCO2', 'Clc1cccc(C[NH2+]Cc2cccc3c2OCCO3)c1', 'Clc1cccc(F)c1O/N=C/c1cc(Cl)c(Cl)c(Br)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnc2c1cnn2-c1ccccc1', 'O=C(/C=C/O)N1CCN(S(=O)(=O)c2c(Cl)cccc2Cl)CC1', 'O=C(/C(Cc1ccccc1)[NH+]1CCCC1)c1ccc([N+](=O)[O-])cc1', 'ClCc1cc([N+](=O)[O-])ccc1OCc1ccccc1F', 'Clc1ccc([C@@H](Cl)[C@]2(F)CCc3ccccc32)cc1', 'Clc1ccsc1C[N-]F', 'Clc1cccc2cc(CNCc3ccccc3F)[nH]c(=O)c12', 'Clc1ccc(-c2cc[nH]n2)c(Nc2ccc(F)cc2Br)c1', 'Clc1cccc(Br)c1O[C@@H]1c2ccccc2C[C@H]1O', 'O=C(CS(=O)(=O)[C@@H]2CCS(=O)(=O)C2)N1Cc2ccccc2C1', 'Oc1cccc([C@H](CNS(=O)(=O)c2ccc(Cl)cc2)[NH+]2CCCCC2)c1', 'Clc1ccccc1NS(=O)(=O)c1cccc(S(=O)(=O)N2CCCCC2)c1', 'OC1=C(c2ccccc2)/C(=C/c2ccc(F)cc2)C1', 'O=[N+]([O-])c1c(Br)scc1[C@H](O)CS(=O)(=O)c1ccccc1', 'Oc1ccc(O[C@H](CC(F)F)c2cccc(Cl)c2)c(F)c1', 'Clc1ccccc1N1N=CC[C@@H]1C(=O)Nc1cccc2ccccc12', 'O=C([O-])c1cc(C2CC2)on1', 'NS(=O)(=O)Cc1noc([C@@H]2CCCN(c3ncccn3)C2)n1', 'Clc1cc(Nc2ccccc2)sc1-c1ccc(Cl)s1', 'Clc1cc(Cl)nnc1N/N=C/c1ccc(Cl)cc1', 'Clc1nnnc(C[NH3+])c1Cl', 'O=C(CS(=O)(=O)[N-]C[C@H]1CCC[NH+]1Cc1cccs1)c1ccc(Cl)cc1', 'C[C@@H]([NH2+]C[C@H](O)C1CCOCC1)c1cccc(OCC#N)c1', 'Clc1nnnn1c1c(O)cccc1N1CCCC1', 'N[C@@H](CC1CCOCC1)C(=O)Nc1ccsc1', 'O=C(CSCN1C(=O)CCc2ccccc21)NCCCc1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])c(Cl)ccc1NC1CC1', 'Clc1cccc2cc(OCn3c(CNC(=O)c4cccc(Cl)c4)no3)cnc12', 'Clc1ccc(-c2csc3[nH]c4cc(Br)cnc4c32)cc1', 'Clc1nnnc(Br)c1I', 'Clc1nnnc(Br)c1I', 'Clc1nncn1c1c(Cl)ccc(Cl)c1Cl', 'Clc1nncc(F)c1I', 'Clc1nncc(F)c1F', 'Clc1nncc(F)c1Br', 'Clc1nncc(Cl)c1I', 'ClCc1cccc(N[C@@H]2CCN(c3ccccc3)C2=O)c1', 'O=C(c1ccsc1)[C@@H]1CC(c2ccc(Cl)cc2)=NO1', 'O=C(c1c[nH]c2nc[nH]c12)N1CCN(c2ccc(F)cc2)CC1', 'O=C([C@H]1c2cc(S(=O)(=O)[O-])ccc2N=C1N1CCOCC1)NNc1ccccc1C(F)(F)F', 'O=C(CSC(/[O-])c1ccc(Cl)c(Cl)c1)NNc1ccccc1Cl', 'Clc1ccc(Br)c(C(=O)/C=C/C2=CCCCC2)c1', 'O=C(COc1cc(Br)ccc1O)NCC(=O)Nc1ccc2n[nH]ccn2c1', 'Clc1ccccc1CNCc1ccc2c(c1)OCO2', 'Clc1nnn(-c2cccc(Cl)c2Cl)n1', 'Clc1ccccc1OCc1csc(-c2ccco2)n1', 'Clc1ccc(C(N)=NC2CCCS2)cc1', 'O=[N+]([O-])c1c([O-])c2sccn2c1C1CC1', 'Clc1ccccc1Oc1cc(N2CCCCC2)ncn1', 'Clc1cc(F)ccc1[C@H](O)c1nncn1C1CC1', 'Clc1nnn(-c2ccc(F)cc2)c1N1CCOCC1', 'Clc1ccc(-c2nn(-c3cc(Cl)ccc3Br)nc2N2CCOCC2)cc1', 'Oc1ccc([N+](=O)c2ccccc2)cc1OSc1ccccc1', 'N=Cc1nc(-c2ccccc2)cs1', 'Clc1ccc([C@@H](NN2CCOCC2)c2cc(F)ccc2F)cc1', 'Clc1ccccc1O/N=C\\c1cccc(Cl)c1', 'Clc1ccc(Br)cc1NCc1ccncc1', 'O=C([O-])/C(=N\\OCc1cccs1)c1ccc(Cl)cc1', 'Cc1c(Br)oc(C)c1C(=O)NC(=S)NC(=O)COc1ccccc1', 'Clc1ccccc1C1=CN(c2ccc3nnnn3n2)CC1', 'CNc1cc[nH+]c(N2C[C@H]([NH3+])C(=O)CC2)c1', 'Oc1ccc(/C=C2/SC(=S)N(c3cccc4ccccc34)C2=O)cc1', 'Oc1ccc(Oc2ccc(NCc3ccco3)cc2)c(Cl)c1', 'O=[N+]([O-])c1ccc(-n2cccc2)cc1O', 'Clc1ccc(-c2cccnc2)nn1', 'COCc1n[nH]c(C)c1[C@@H](C)NC(=O)c1ccc(OC)c(F)c1', 'O=C(CNS(=O)(=O)/N=C/c1ccccc1)NCCc1ccc(Cl)cc1', 'Oc1cc(OCOc2ccc(Br)cc2)[nH]n1', 'OCc1ncn[n-]1', 'OCc1nc[nH]n1', 'OCc1nc[nH]n1', 'Clc1cccc(S(=O)(=O)Nc2ccc3c(c2)OCCO3)c1O', 'Clc1ccc(Br)c([C@H]2OCC[C@@H]2CO)c1O', 'Oc1ccc(/C=C/N2CCOCC2)cc1', 'Clc1ccc(-c2cc(NCc3cccs3)no2)cc1', 'Oc1cccc2nn[nH]c1c1ccccc12', 'Oc1cccc2nn[nH]c12', 'Oc1cccc2nn[nH]c12', 'Oc1cccc2cc(/C=C/C(=O)c3ccc(F)c(F)c3)cnc12', 'O=C(CNS(=O)(=O)c1ccc(Cl)s1)Nc1cc(Cl)c(Cl)cc1Cl', 'Clc1ccc(Br)c(NC[C@@H]2NCCC[NH+]2CC2CC2)c1', 'Clc1cccc(F)c1Br\n\n\n\n', 'Clc1cccc(F)c1Br\n\n\n', 'Clc1cccc(F)c1Br\n\n', 'Clc1cccc(F)c1Br\n', 'Clc1cccc(F)c1Br', 'Clc1cccc(F)c1Br', 'Clc1cccc(Br)c1Cl\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n\n\n\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n\n\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n', 'Clc1cccc(Br)c1Cl\n\n', 'Clc1cccc(Br)c1Cl\n', 'Clc1cccc(Br)c1Cl', 'Clc1cccc(Br)c1Cl', 'Clc1cccc(Br)c1Br\n\n\n\n\n\n\n\n', 'Clc1cccc(Br)c1Br\n\n\n\n\n\n\n', 'Clc1cccc(Br)c1Br\n\n\n\n\n\n', 'Clc1cccc(Br)c1Br\n\n\n\n\n', 'Clc1cccc(Br)c1Br\n\n\n\n', 'Clc1cccc(Br)c1Br\n\n\n', 'Clc1cccc(Br)c1Br\n\n', 'Clc1cccc(Br)c1Br\n', 'Clc1cccc(Br)c1Br', 'Clc1cccc(Br)c1Br', 'Nc1nonc1Cc1ccc[nH]1', 'ClCc1nnn(-c2ccccc2)n1', 'ClCc1ccnnc1-n1cncn1', 'Clc1ccccc1[C@H]1N=NC[C@H]2COCCS[C@@H]21', 'O=[N+]([O-])c1cc(F)c(OC[C@@H]2CCCCO2)cc1Br', 'O=C(COc1ccc2nsnc2c1)N[C@@H]1CCCC[C@H](O)C1', 'Oc1cc(-n2cc[nH+]c2)ccc1NC(=O)[C@@H]1C[C@H]2C=C[C@@H]1C2(Cl)Cl', 'Clc1ccsc1SCc1ccc(C(=O)[O-])cc1', 'Clc1ccccc1Oc1cc(C(F)(F)F)nc(Nc2ccc(F)cc2)n1', 'Clc1ccc([C@@H](Cl)Cc2ccc(F)cc2)cc1Cl', 'Clc1ccc(Br)cc1Sc1ccccc1', 'Clc1ccc(Br)cc1Sc1ccccc1', 'Clc1cc(Br)ccc1Sc1ccccc1', 'Oc1cc(F)cc(NC2(c3cccc(F)c3)CCC2)c1', 'Clc1nnn(-c2ncc(C(F)(F)F)cc2Cl)c1Br', 'Cc1ncsc1CO[C@@H]1CCC[C@@H](C)C1', 'CNS(=O)(=O)[C@@H](C)c1ccccc1', 'Cc1cncc(NC(=O)c2nnc(-c3ccc4ccccc4n3)s2)c1', 'N=Cc1cncc(CN2CCCC[C@@H]2c2cccc([N+](=O)[O-])c2)n1', 'NS(=O)(=O)/C(C#N)=C/c1ccc(Oc2ccc(F)cc2)cc1', 'OCSc1nc2ccccc2[nH]1', 'Clc1cc(F)c(Cl)cc1NCCc1nc(Cl)ns1', 'Clc1ccccc1Oc1cnc(CSc2ncccn2)nc1', 'Clc1ccc(-c2cc[nH]n2)c([C@H]2CC[NH2+]C2)n1', 'O=C(/N=C\\c1ccco1)[C@@H]1CCCc2sccc21', 'Clc1cccc2cc(F)ncc12', 'Clc1cccc2cc(F)ncc12', 'Clc1cccc2cc(Br)cnc12', 'Clc1cccc2cc(Br)cnc12', 'N#CCNC(=O)c1cc(C(=O)N2C[C@H]3CCCC[C@@H]3C2)nn1-c1ccccc1', 'N[C@@H]1N=Nc2nnnn21', 'Clc1ccc(Br)cc1CCn1cccn1', 'ClCc1cncn2cnnc12', 'O=C(CNS(=O)(=O)C(F)(F)F)Nc1ccccc1Br', 'Cc1nccn1-c1cccc(C[C@H]2CCC[NH2+][C@H]2C)c1', 'O=[N+]([O-])c1c(Cl)nccc1C1CC1', 'Clc1cccc2nn(S[C@@H](NC3=CCCC3)C(F)(F)F)nc12', 'Oc1ccc(/C=C(\\Cl)CS(=O)(=O)c2cccc(Cl)c2)cc1', 'Clc1ccccc1NC1=NN([C@@H]2CCS(=O)(=O)C2)C1', 'O=C(c1cc(Cl)cn2cnnc12)N1CCN(Cc2ccccc2)CC1', 'Clc1cc(F)cc2nc(CC3CCN(c4ccccc4F)CC3)nn12', 'Oc1cc(F)cc2c1cnn2Cc1cccs1', 'ClCc1cn(C[C@@H]2CCCO2)c(C(=O)[O-])c1', 'Clc1ccc(Br)c(Sc2ccc(OCc3ccccc3)cn2)c1', 'O=[N+]([O-])c1c([O-])sc(OC2CCCCC2)c1[N+](=O)[O-]', 'Oc1ccc([C@H]2CN(Cc3csc(Cc4cccc(F)c4)n3)CC2)cc1', 'Clc1ccsc1-c1cc(Br)ccn1', 'ClC(=S)NNc1ccccc1[N+](=O)[O-]', 'Clc1nnn(C2CCCC2)c1-c1ccc(Br)s1', 'C=CCn1cc(NC(=O)c2cc(C(C)C)ncc2F)cn1', 'N=Cc1nonc1Cc1cc(C)ccc1OCC(F)(F)F', 'Clc1ccccc1[C@H]1Nc2c(cccc2Cl)C(=O)N1C1CCCCC1', 'O=C(c1cn2ccsc2n1)N1CC[C@](N2CCOCC2)C1', 'Nc1nonc1SCc1ccccc1OCc1ccc(Cl)cc1', 'Clc1cccc(-c2ccoc2Br)c1', 'Nc1[nH]nc(Cn2cc(-c3ccc(Cl)cc3)cn2)n1', 'Oc1cccc2nnc(-c3ccccc3F)n12', 'Oc1ccccc1C(=O)OCc1ccc(F)cc1', 'N#CC[C@]1(O)CCCN(C(=O)c2cn(Cc3ccccc3)n[nH+]2)C1', 'Clc1cccc2nn(CCCCNc3cccc(Cl)c3)nc12', 'Clc1ccc(-c2nonc2[S-])nc1', 'Clc1ccc(-c2nonc2[S-])nc1', 'Clc1ccsc1N/C1=C2C=C[C@H](C2)c2ccccc21', 'ClC[C@@H]1OCCOc2ccccc21', 'Clc1ccccc1Oc1cncc(Br)c1', 'Clc1ccccc1Oc1cc(Cl)ccn1', 'Clc1ccccc1Nc1cc(Cl)ccn1', 'Clc1ccc(Br)cc1-c1cnccn1', 'Clc1ccc(Br)cc1-c1cnccn1', 'Clc1ccc(-c2cc[nH]n2)c(F)c1', 'Clc1ccc(-c2cc[nH]n2)c(Cl)c1', 'Clc1ccc(-c2cc(O)n[nH]2)cc1', 'Clc1ccc(-c2cc(Cl)ncn2)cc1', 'Clc1cc(Cl)nnc1-c1ccccc1', 'Clc1cc(Cl)ncc1Oc1ccc2ccccc2c1', 'Clc1cc(Cl)cnc1[N-]c1ccc2ccccc2c1', 'Oc1cc(O)c2nonc2c1N[C@H]1CC[NH2+]C1', 'O=C([C@H](c1ccccc1)n1cccc1)c1cccc(Cl)c1', 'Oc1cccc(OC[C@H]2CCCO2)c1', 'Oc1cccc(O[C@@H](C(=O)[O-])c2ccc(Oc3ccccc3)cc2)c1', 'Clc1nnn(-c2cncc(C[NH+]3CCOCC3)c2)c1Cl', 'Clc1cc(Cl)nnc1S[C@@H](C[NH3+])c1ccccc1', 'O=C(Cc1nn(-c2ccccc2)c([O-])c1Cl)Nc1cccc([N+](=O)[O-])c1', 'C[C@@H]([NH2+][C@H](CO)c1ccccc1Cl)c1cccc(F)c1', 'O[C@@H](Sc1nncn1Cc1ccccc1)Cc1nnc2ccccn21', 'O=[N+]([O-])c1ccncc1CN1C(=O)c2ccccc2C1=O', 'O=C(CSCN1C(=O)/C(=C/c2ccccc2)SC1=S)NCc1ccc2c(c1)OCO2', 'Clc1cc(/N=C\\c2sccc2CO)sc1NCc1ccc(Cl)cc1', 'Cc1snnc1S[C@H](C)C(=O)N(C)Cc1ccc(=O)[nH]c1', 'Clc1ccccc1O/N=C\\Nc1c(Cl)cccc1Cl', 'NS(=O)(=O)N1C(=O)/C(=N/c2nc(-c3ccccc3Br)n[nH]2)c2ccccc21', 'N#C[C@H](C(=O)Cc1cccc(Cl)c1)CCS(=O)(=O)c1ccccc1', 'Clc1ccc(Br)cc1-c1ccsn1', 'Clc1ccc(-c2n[nH]c(=S)n2CN2CCOCC2)cc1', 'Clc1cccc([S@@]CCOc2ccc(Br)cn2)c1', 'O=C(CNc1cccc(OC(F)(F)F)c1)N1CCN(S(=O)(=O)c2ccccc2)CC1', 'Clc1ccc(C2(F)CC2)cc1', 'ClCc1csc2nc(-c3c[nH]c4ccccc34)oc12', 'O=C(/C(c1cccs1)=[NH+]/Nc1cccc(C(F)(F)F)c1)N1CCN(c2cccc(Cl)c2)CC1', 'O=[N+]([O-])c1cc(Cl)ncc1NC[C@H](SCc1ccccc1)c1ccccc1', 'O[C@@H]1c2ccccc2C(=O)N1Cc1cccs1', 'Clc1ccc(Br)cc1-c1cnn(-c2ccc(F)cc2)n1', 'Clc1ccc(Br)cc1-c1cnn(-c2ccc(F)cc2)n1', 'Oc1cccc(S[C@@H]2C[C@@H](O)[C@@H](O)[C@@H]2O)c1', 'Oc1ccc([C@H]2OC[C@@H](O)C[C@@H]2C[NH2+]Cc2cccc(Br)c2)cc1', 'O=C(Cc1nonc1C(F)(F)F)C1CCN(S(=O)(=O)c2ccccc2)CC1', 'Clc1cccc2c(N)cc(C[C@@H]3CN4S(=O)(=O)CCC[C@@H]4O3)cc12', 'Oc1cc(Br)c(-c2ccncc2)c(Br)c1', 'Cc1snnc1CN1CCC(C(=O)NC2CCCCC2)CC1', 'Clc1ccccc1Cn1cnnc1SCc1ccccc1', 'ClC(=S)[N+]1(c2cc(Br)ccc2F)CC1', 'ClCc1csc(-c2ccccc2F)n1', 'ClCc1ccn(-c2cccc(Br)c2)n1', 'OCc1n[nH]c(Nc2cccc(-c3ccccc3)c2)c1C1CC1', 'COc1nn(-c2c(C)cc(C)c(C)c2NC(=O)C(=O)N[C@@H]2CCCC[C@@H]2C)cn1', 'OC[C@@]1([NH3+])Oc2ccccc21', 'ClC([O-])[C@@]1(O)Cc2ccccc21', 'NS(=O)(=O)/C=C(/Cc1ccc(O)cc1)c1ccccc1', 'Oc1ccc([N+](=O)[O-])c(S[C@@H](C(=O)[O-])c2ccccc2Cl)c1', 'O=[N+]([O-])c1c(Cl)nc2[nH]ncc2c1Cl', 'Clc1nnnc(Cl)c1-n1cccn1', 'Clc1nnn(-c2ncccn2)c1Br', 'O=C(/N=C\\c1ccc(S(=O)(=O)N2CCCC2)cc1)c1ccccc1Cl', 'Clc1ccc(-c2cc[nH]n2)nc1Sc1ccccc1', 'Clc1ccc(-c2cc(-c3cnc([C@H]4CCCO4)nn3)[nH]n2)cc1Cl', 'Clc1ccc(C(F)(F)F)cc1N1CCCC[C@H]1CN1CCOCC1', 'Cc1ccc(C(=O)CC[n+]2cccc[nH+]2)cc1F', 'NS(=O)(=O)C[C@@H]([N+](=O)[O-])c1cccc2ccccc12', 'C=C(C(=O)CN1C(=O)COc2ccc(S(=O)(C)C)cc21)c1ccccc1', 'ClCc1cc(C(F)(F)F)nc(N2CCOCC2)n1', 'Clc1ccc(Cc2c[nH]nc2-c2ccccc2)c(Cl)c1', 'O=C(Cc1cc(Cc2ccccc2)no1)c1cccc(Cl)c1', 'N#C[C@@H]1OC(=O)N(CCCN2CCc3ccccc32)C1=O', 'NS(=O)(=O)CC(=O)Nc1ccc(C(=O)NCc2ccco2)cc1', 'Clc1ccccc1C1CCN(S(=O)(=O)c2cc(OC(F)(F)F)ccc2Cl)CC1', 'O=[N+]([O-])c1ccc(-c2noc(-c3ccc([N+](=O)[O-])cc3)n2)cc1Cl', 'Clc1nnn(C[C@@H]2CCc3ccccc32)c1Cl', 'Nc1cn[nH]c1', 'Clc1ccccc1OCc1csc(C(=O)[O-])n1', 'O=[N+]([O-])c1c(Br)sc2c1C(=O)[C@@H](c1ccccc1)[C@@H]1CC=C[C@H]21', 'Clc1ccccc1N1N=C(c2ccco2)CC1', 'O=C(COc1cc([N+](=O)[O-])cc(C(=O)[O-])c1)N/N=C/c1cc(Br)ccc1F', 'O=[N+]([O-])c1ccn(Cc2cccc(C(=O)N3CCCC3)c2)c1', 'NS(=O)(=O)Cc1nc(Oc2ccc(Cl)cc2)c2c(-c3cccc(Cl)c3)ccc([O-])c2n1', 'Oc1cccc2nncn12', 'Clc1ccc(Cc2cccc(O)c2)nc1', 'Clc1nnnn1/N=C/c1ccc([N+](=O)[O-])cc1', 'Clc1nnc([S-])n1CCc1ccccc1C(F)(F)F', 'O=C(Cc1c(Cl)cccc1Cl)Nc1c(Cl)ccc2cccnc12', 'Clc1nnc(C[NH+](C2CC2)[C@H]2CCS(=O)(=O)C2)s1', 'Clc1cccc(CCc2nnc(Cc3ccsc3)n2Cc2ccccc2)c1', 'O=C(Cc1coc2ccccc12)NCc1cccc(-n2cccn2)c1', 'N[C@@H](Cc1ccc2c(c1)C(=O)CCC2)c1cc(OC(F)(F)F)ccc1Cl', 'Cc1c(C(=O)N[C@H](C(=O)N2CCCC2)nncc1C(=O)N1CCCCC1)c1ccccc1', 'Clc1ccsc1N[C@@H]1c2ccccc2C[C@@H]1O', 'Clc1cccc(-c2n[n-]c(-c3ccccc3)n2)c1', 'O=C(C1=C(NC2=NN=C(SCc3ccccc3F)N=N2)[C@@H]2CS(=O)(=O)C=C12)Nc1ccc(Br)cc1', 'Clc1cc(Cl)c2sccc2n1', 'Cc1cncc(C(C)C)c1NC(=O)[C@@H](C)Oc1ccccc1C#N', 'Oc1cccc([C@H]2OC(/C=N/OC(=O)c3ccccc3)c3ccccc32)c1', 'Clc1ccsc1S(=O)(=O)C1CC[NH2+]CC1', 'Clc1ccc(Br)cc1-n1cnc(Cc2ccccc2)n1', 'O=[N+]([O-])c1cc(C(=O)Nc2ccc(Cl)cc2)ccc1N/N=C/[C@@H]1C=CC=C1Cl', 'Clc1ccc(-c2cccnc2N=[N+]2CCCC2)cc1', 'Oc1ccc([N+](=O)[O-])c(CSc2nnc[nH]2)c1Cl', 'O=C(Cc1cccc(Cl)c1)NCc1cccc(-c2ncnn2C2CC2)c1', 'Clc1ccc(-c2ccccc2CCSCc2ccc(F)cc2F)cc1', 'Clc1nncnc1C(=O)Nc1ccc(F)cc1', 'O[C@@H](COCc1cc(-n2cccc2)c[nH]1)c1ccc(F)cc1', 'O=[N+]([O-])c1ccc2oc([C@H]3CCC[NH2+][C@@H]3Cc3ccc(F)cc3)cc2c1', 'N[C@@H](Cc1c[nH]c2ccccc12)C(=O)Nc1cccc(Cl)c1', 'O=C(c1cc([S@](=O)CC(=O)Oc2ccccc2F)cs1)[NH+]1CCCC1', 'Clc1cccc2ccc(OCCSc3nnnn3-c3ccccc3)c2c1', 'Clc1nnnc(Cl)c1I', 'Clc1nnnc(Br)c1Br', 'Clc1nncc(Cl)n1', 'Clc1nncc(Cl)n1', 'Clc1nncc(Cl)n1', 'Clc1nnc(Cl)nc1F', 'Clc1nnc(Br)cn1', 'ClC[C@H](C[C@@H]1CCCO1)c1ccc(Br)s1', 'Nc1[nH]nc(N2CCC[C@@H](c3nc4ccccc4[nH]3)C2)c1N', 'Clc1ccc(-c2csc3[nH]ncc23)cc1', 'Clc1ccc(-c2csc3[nH]cnc32)cc1', 'Clc1ccc(-c2csc3[nH]cnc23)cc1', 'Clc1ccc(-c2csc3[nH]cnc23)cc1', 'Clc1cccc2[nH]nc(S/c3ccc(F)c(F)c3)c12', 'ClC[C@H](Br)c1c(Cl)cccc1Cl', 'Cc1cnc2oc(C)c(C(=O)N[C@H]3CC(=O)N(c4ccccc4F)C3)c2c1', 'O=C(c1cccnc1)N1CCc2cc(Cl)ccc21', 'Clc1ccc(COC[C@@H](O)c2ccccc2)cc1', 'Clc1ccccc1Oc1cc(OCc2ccc(Cl)cc2)cc2c1OCO2', 'Clc1cc(F)cc2nc[nH]c12', 'Clc1cc(Cl)cc2[nH]ncc12', 'Clc1cc(Br)cc2[nH]cnc12', 'Oc1ccc(F)cc1CN1CC[NH+]c2ccccc2c1=O', 'C=CCCn1/c(=N/C(=O)c2ccccc2)sc2cc(C)c(C)cc21', 'C=CCC[C@H]1CCC[NH+](Cc2cccc(NC(C)=O)c2)C1', 'Clc1ccccc1-c1cnnn1[C@@H]1CC[C@](O)(C(F)(F)F)C1', 'Clc1ccccc1Nc1cnc(I)c(Br)c1', 'Clc1ccccc1Nc1cnc(Cl)cc1I', 'OC[C@H]([NH2+]Cc1ccccc1)c1cc(Cl)sc1Cl', 'O=C(/C=C\\c1cccnc1)NNc1cccc(NC(=O)c2cccnc2)c1', 'Clc1ccc(Br)c(F)c1NC1[C@@H]2CC[C@@H]1C2', 'Clc1cc(Br)cnc1[S-]', 'Clc1cc(/N=C\\c2c([O-])[nH]c3ccccc32)[nH]c(=O)c1Br', 'O=C(c1cc(Cl)cnc1Cl)N1CCC(Oc2ccccc2)CC1', 'Oc1cc(-n2ncc(-c3ncnn3CCOc3ccccc3)n2)cc(S[C@H]2CCOCC2)c1', 'Oc1cccc(Nc2nnc(SCc3nc4cccc(F)c4s3)n2N2CCOCC2)c1', 'ClC[C@@H]1Oc2ccccc2C[C@@H]1O', 'Clc1cc(/C=C2\\S/C(=C/c3ccc(O)c(OCc4ccccc4)c3)C(C#N)=C2[O-])ccc1O', 'Oc1ccc(F)c(O[C@H]2CCN(Cc3ccccc3)C2=O)c1', 'Cc1sc2sc(-c3ccc(S(=O)(=O)NC(C)C)cc3)nc2c1C', 'Clc1ccccc1NS(=O)(=O)c1c[nH]c2ccccc12', 'Clc1cc(CCl)c[nH]c1=O', 'N#Cc1cc(Br)cc(NC[C@@H](O)COc2ccc(Br)cc2)c1', 'Clc1cc(/C=C(/NC2=NN=CC2)Sc2ccccc2)[n+]([O-])c2ccccc12', 'Oc1cc(O[C@@H](C#N)c2ccccc2)ccc1CN1CCOCC1', 'Clc1ccc(-c2cc(F)c(F)c(F)c2COc2ccc(CC(=O)[O-])cc2)cc1', 'O=C([O-])c1nc(CSc2cc(F)ccc2F)c(F)cc1F', 'ClCc1cn[nH]c1-c1ccccc1C(=O)Nc1ccc([N+](=O)[O-])cc1Cl', 'C=C[C@@H](O)CN1CCc2ccc(C(=O)NC)cc2C1', 'Oc1ccc([N+](=O)[O-])c(-n2cccc2)c1', 'O=[N+]([O-])c1ccnc2ccc(F)c(-c3ncoc3C(F)(F)F)c12', 'Clc1ccc(Br)c(/[NH+]=C\\c2ccccc2)c1', 'Clc1ccc(Br)c(/[NH+]=C/c2ccccc2)c1', 'COc1nn(C)cc1C(=O)Nc1nc(-c2ccccc2)c(C)s1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1-c1ccc(Cl)cc1', 'Clc1ccccc1Nc1nn[n-]c1C(=O)[O-]', 'O=C(c1c[nH]nc1-c1ccoc1)N1CC[C@@H](c2ccc(F)c(F)c2)C1', 'Clc1ccc(Br)c([C@H]2CC[NH2+]C[C@@H]2c2cccc(Br)c2)c1', 'O=[N+]([O-])c1c([O-])[nH]c2nccnc12', 'C=C1NC(=[NH2+])[C@@H]1/C=N/NC(=O)C(=O)N/N=C/c1ccc(C)cc1', 'Clc1cccc(NCCS(=O)(=O)/N=C/c2ccccc2)c1', 'Clc1cc(Br)c2nc(-c3ccccc3)[nH]c2c1', 'Clc1ccccc1-c1csc(C2CCCC2)n1', 'Clc1ccc(-c2nnnn2-c2cccc(Cl)c2)cc1', 'O=[N+]([O-])c1c(Cl)n2ncnc2n1Cc1ccccc1', 'Clc1ccc(-c2nonc2Cn2cncn2)cc1', 'ClCc1cc(F)cnc1Oc1ccccc1', 'Oc1ccc([N+](=O)[O-])c(/C=C/Nc2ccccc2Br)c1', 'O=C(/C(c1cccc(Cl)c1)N1CCOCC1)c1ccc(C(F)(F)F)cc1', 'O=[N+]([O-])c1cc(N2CCN([C@@H]3Cc4ccccc4O3)CC2)ccc1Cl', 'Clc1cc(NSc2ccc(F)c([N+](=O)[O-])c2)ncn1', 'Oc1ccccc1-c1csc2nc(SCC(=O)OC[C@@H]3CCCO3)[nH]c12', 'CCCCc1ccc(CNC(=O)NC(C)(C)c2cccc(C)c2)s1', 'O=C([O-])[C@H](NS(=O)(=O)c1ccc(Cl)cc1)c1cscc1C(F)(F)F', 'Oc1cc(Br)cc2cn[nH]c12', 'Clc1ccc(Br)cc1OS(=O)(=O)Cc1ccccc1F', 'Clc1ccccc1NC(=S)NNc1ccc(F)cc1F', 'O=C(c1ccco1)[C@H]1Sc2ccccc2NC(=O)[C@H]1Cc1cccc2ccc(Cl)cc12', 'ClC(F)(CCO)N1CCc2sccc2C1', 'Oc1cccc(-c2ccnc(NCCOc3ccccc3)c2)c1', 'O=C(c1cccc([N+](=O)[O-])c1)N1CCN(Cc2ccccc2)CC1', 'Oc1ccc(C(=O)c2sc(N3CCOCC3)cc2C(=O)[O-])cc1', 'COc1ncccc1N(CC)Cn1nc(C)cc1C', 'Clc1ccc(Br)cc1/C=C(/C#N)C(=O)N[C@@H]1CCS(=O)(=O)C1', 'Cc1cc(Cl)ccc1NC(=O)CSC1=N[C@@H]2CS(=O)(=O)C[C@@H]2N1c1ccccc1', 'Nc1c(Oc2nc(N)nc3ccccc23)cccc1I', 'Oc1cc(-n2cc(C(F)(F)F)ccc2=O)ccc1NC[C@H]1CCCC[C@@H]1O', 'Oc1cc(-n2ncc(Br)c2Cl)ccc1-c1ccccc1O', 'COc1nn(-c2cccc(C(=O)Nc3cc(Cl)c(OC)cc3)c2)c(C)c1Br', 'Oc1cc(F)ncc1S[C@@H](CO)Cc1c[nH]nc1C1CC1', 'Clc1cc(Br)cnc1N1CCOCC1', 'Clc1ccsc1S(=O)(=O)N1CC[C@H]2CCCC[C@@H]2C1', 'O=C([O-])[C@@H](Cc1ccccc1)c1cc(Cl)ccc1F', 'Clc1cccc([S@@]c2ncc(Br)cc2Cl)c1', 'Clc1cc(Br)c(Sc2cccc(F)c2)nc1', 'Clc1cc(Cl)nn1/N=C\\c1ccc2c(c1)OCO2', 'Clc1nncnc1/C=C/C(=O)Oc1ccc(Cl)cc1', 'Clc1ccc(CC[C@@H]2CCC[NH+](CC3CC3)C2)c(O)c1', 'Clc1cc(F)c(NCCCC[NH2+]C2CC2)cc1', 'Clc1ccccc1Oc1cnc2nn[n-]c2c1', 'OCc1n[nH]c(N/N=C/c2ncc(Br)cn2)n1', 'O=C(/N=C\\NCc1nc2ccccc2s1)c1cccc2ccccc12', 'Oc1cc(COc2ncnc(-c3cccs3)n2)ccc1O', 'O=C(CSCC(=O)N1CCCCC1)NNC(=O)c1ccccc1C(F)(F)F', 'O=C([C@H](Cl)c1ccccc1)c1cc(Br)ccc1F', 'Clc1cc(Cl)nn1/N=C\\c1cccc(Br)c1', 'Cc1c(-c2cncc(NC3CCCC3)c2)oc2ccccc12', 'Clc1ccsc1N1CCCCC[C@@H]1C(=O)[O-]', 'Clc1ccc(Br)cc1CC[NH+]1CCCC1', 'Clc1cc(C[NH2+]C2CC2)ccc1N1CCOCC1', 'Clc1ccc(-c2csc3s/cnc23)cc1Cl', 'Clc1cccc2ncn(C3CC[NH+]=C3C(=O)NCCN2c2ccccc2Cl)c1', 'Oc1cc(C=O)nc(-c2ccccc2)n1', 'Clc1ccc(-c2cc([C@H](Cl)Cc3ccc(F)cc3)[nH]n2)cc1', 'Oc1ccc(Oc2ccc3ccccc3n2)cc1Cl', 'Nc1nncn2cnnc12', 'COc1ncnc(CNc2cccc(OC(C)C)c2)c1[N+](=O)[O-]', 'Cc1nonc1NC(=O)N1CCN(c2cc(C)ccn2)CC1', 'Clc1ccc(C(N)=N\\O[C@H]2CCO[C@]3(CCOC3)C2)cc1', 'O=C(c1ccsc1)N(CCNS(=O)(=O)c1ccccc1)N1CCOCC1', 'Clc1cc(F)cc(SCc2nc(Cc3ccccc3)cs2)c1', 'Oc1ccc([N+](=O)c2cc(COC3CCCC3)c[nH]2)cc1', 'Cc1c([N+](=O)[O-])cccc1N1C(=O)[C@H]2SCC[C@@H]12', 'Oc1cccc(Br)c1Cn1cccn1', 'Clc1ccc(-c2cccs2)nc1C[C@H]1CCCCN1', 'ClCc1cccc(CO)c1I', 'Clc1ccc(Br)cc1CC[NH2+]C1(c2ccc(F)cc2)CCOCC1', 'O=[N+]([O-])c1c(-c2c[nH]c[nH+]2)[nH]c2c1cnn2-c1cccc(Br)c1', 'Clc1cc(/C=C(\\F)C(F)(F)F)ccc1Cl', 'O=[N+]([O-])c1cc(F)ccc1C[NH+]1CCC(c2ccccc2)=N1', 'NS(=O)(=O)Cc1ccccc1C(=O)N1CCC[C@@H]1C(N)=O', 'ClC[C@@H]1CO[C@H](c2ccccc2F)N1', 'Clc1ccc(C[NH2+]Cc2cc3ccccc3o2)cc1', 'Clc1ccc(C2=CC[C@@H]2C2CCC2)cc1F', 'OCc1n[nH]c(Nc2cccc3ccccc23)n1', 'Clc1cc(/C=N/OCc2ccccc2)co1', 'Clc1ccc(-c2cc(Br)cnc2Cn2ccnc2)cc1', 'Clc1ccc(-c2n[nH]c(C(=O)N3CCCC3)c2)cc1', 'Clc1cc(Nc2ncnc3c2CCCCC3)ccc1Cl', 'Clc1cc(Cl)cnc1-c1ccc2c(c1)COCO2', 'Clc1cc(Cl)c(Cl)nc1N[C@@H]1CCOc2ccccc21', 'O=C(c1c[nH]nc1-c1ccc2c(c1)COCO2)c1ccccc1Cl', 'ClCc1n[nH]c2ccc(F)cc12', 'ClCc1ccc2nc[nH]c(=O)c2c1', 'N#CCSc1ccccc1Cn1c(=O)[nH]c2ccccc21', 'N#CCc1ccc(OCC[NH+]2CCC(CN3C[C@H](C)O[C@H](C4CC4)C3)CC2)cc1', 'Clc1cccc(-c2noc(CSc3ncnc4[nH]ncc34)n2)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1N1CCOCC1', 'Oc1cc(-n2nc3c(c2)CCCN3C(=O)c2ccc(Cl)cc2)cc(Cl)c1', 'Clc1ccc(Cc2nnn[n-]2)cc1', 'O=C(/C(c1ccccc1)[C@@H]1CC=CCC1)c1ccc([N+](=O)[O-])cc1', 'O=C(C1=CC(=N/O)CC1)c1ccccc1', 'O=[N+]([O-])c1ccc(CN2CCc3[nH]c[nH+]c3/C(=N/OCc3cccs3)C2)cc1', 'N=Cc1nonc1-c1ccc(OC(F)F)c(C)c1', 'Clc1cc(Nc2ncc[nH]2)nc2ccccc12', 'ClC(F)(CCC(F)F)NCc1cc(Cl)cs1', 'OC[C@@H]1[C@H](OCc2ccccc2)c2ccc(F)cc2O1', 'Clc1ccc(-c2cc(N[C@@H]3CCS(=O)(=O)c4ccccc4C3)ccn2)cc1', 'Clc1ccccc1NS(=O)(=O)NCc1cccc([N+](=O)[O-])c1', 'Oc1ccc([N+](=O)[O-])c(Cl)n1', 'Oc1ccc([N+](=O)[O-])c(Cl)n1', 'Clc1nnnc(O)c1Br', 'Clc1nnnc(Cl)n1', 'Clc1nnnc(Cl)n1', 'Clc1nnnc(Br)n1', 'O=C(/N=c1/[nH]c/nc2ccc(Br)cc2c1=O)c1ccc(C2CCCC2)cc1', 'Clc1ccsc1-c1csc(N2CCCCCC2)n1', 'Oc1cccc2nnc(S(=O)(=O)NC[C@]3(O)[C@@H]4C=CC[C@@H]4C3)n12', 'Oc1ccc(/C=N\\NC2=NC=NC3=C2SCC3)cc1', 'Clc1ccccc1C(F)(F)F', 'Clc1ccc(-c2cc(S(=O)(=O)N3CCN(c4ccccc4)CC3)co2)cc1', 'O=[N+]([O-])c1c(-c2ccccc2)sc(SCC(=O)N2CCOCC2)n1', 'Clc1cccc(-c2cccc(OCc3ccccn3)c2)c1', 'Nc1nc(-c2ccc(Cc3cccs3)nc2)ccc1C(F)(F)F', 'Nc1c(Cl)ncn2ccnc12', 'Clc1nncnc1N[C@H]1CCc2ccccc21', 'O=C(c1ccsc1)c1n[nH]c2ccccc12', 'Clc1nncc(OC[C@@H]2Cc3cc(F)cc(F)c3O2)n1', 'Clc1cc(N2CC[C@H](Nc3ncccn3)C2)ncc1Cl', 'Oc1cc(-n2nc3c(=S)[nH]c(-n4cncn4)c3c2=O)ccc1Cc1ccccc1F', 'Clc1cc(NC[C@@]2(O)CCSC2)ccn1', 'O=C(CNc1cc2n(n1)CC[NH2+]C2)c1cccs1', 'Oc1ccccc1[C@H]1ON=C(/C=N\\c2ccccc2Cl)[C@@H]1C(=O)[O-]', 'Oc1cccc(CCCC(=O)c2ccccc2F)c1', 'Clc1cccc([S@@]CCn2ccnc2OCC(F)F)c1', 'Clc1cccc(/N=N/[C@H]2c3cc(OCc4ccccc4)ccc3C2=O)c1', 'Clc1cccc(-c2nc3cc(Cl)ccc3o2)c1', 'Nc1nccs1', 'Oc1cccc2cc(-c3ccccc3)cnc12', 'N#C[C@@H]1CN(Cc2ccccc2Cn2cccn2)CCS1', 'O=[N+]([O-])c1cc(C2SCCS2)c2ccccc2c1[N+](=O)[O-]', 'O[C@@H](c1csc(C2CCCCC2)n1)[C@H]1CCOC(O)(CO)C1', 'Oc1ccccc1/N=N\\n1cnnc1', 'O=[N+]([O-])c1c(Br)cnc2nccn12', 'Clc1ccccc1NC1COC1', 'Oc1cc(OC(F)F)ccc1-c1ccccc1', 'Oc1cccc(-n2c(SCC(=O)N3CCCC3)nc3ccccc32)c1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1N[C@H]1CCCOc2ccc(F)cc21', 'OCc1nnnn1-c1ccccc1', 'Clc1cc(/N=C/c2ccc(-c3ccco3)cc2)cs1', 'ClC(/C=[NH+]/CC[NH+]1CCCc2ccc(C(F)(F)F)cc21)c1ccc(Cl)cc1', 'NS(=O)(=O)N(C)c1ccccc1OCc1nc2ccc(Br)cc2[nH]1', 'Clc1ccc([C@@H](O)[C@]2(O)CCc3cccnc32)cc1', 'O=C(/C([O-])(c1ccc(F)cc1)c1cnc(Cl)cc1)[C@@H]1SCCS1', 'Clc1ccc(-c2cc([C@H]([NH3+])C3CCCC3)cnc2C#N)cc1', 'Oc1ccc([N+](=O)[O-])c(Cl)c1OC[C@H]1CCCO1', 'O=C(CNS(=O)(=O)/c1c(Cl)cccc1Cl)NCCc1ccccc1', 'Clc1ccccc1-c1nc(C[S@@](=O)C/C=C/c2ccc(Br)cc2)no1', 'Oc1cccc(OC[C@@H]2CO2)c1', 'Clc1nncnc1N', 'Oc1ccccc1O[C@@H](CC(F)(F)F)c1cc(F)cc(F)c1', 'Clc1ccc(-c2nonc2)nc1C1CC1', 'Clc1ccc(-c2cc(Br)c[nH]c2=O)cc1', 'ClCc1csc(OCc2ccc(Cl)cc2Cl)n1', 'CNS(=O)(=O)CCCNC(=O)/C=C/c1cccc2ccccc12', 'O=C(/C(Cc1cccs1)Nc1cccc2ccccc12)c1cccnc1', 'C=CCn1nc(SCC(=O)c2ccccc2)ccc1=O', 'Clc1nncn1/c1nc2ccc(Br)cc2o1', 'Clc1ccc(Br)cc1OS(=O)(=O)C[C@@H]1COc2ccccc2O1', 'Oc1ccc([N+](=O)C2=C([O-])O[C@@H]2c2ccccc2)cc1OCC(F)(F)F', 'Clc1cccc2nnnc2c1COc1cccc2c1OCO2', 'Nc1cn[nH]c(=O)c1[N-]S(=O)(=O)c1ccc([N+](=O)[O-])cc1', 'Clc1ccccc1-c1csc(-c2ccco2)n1', 'Clc1ccc(-c2cccs2)nn1', 'Clc1ccc(-c2cccs2)nn1', 'Clc1ccc(-c2cccs2)nn1', 'Clc1ccc(-c2cccs2)c2nccnc12', 'Oc1cc(-n2nnnc2SCc2ccccc2)c2ccccc2n1', 'Clc1nnc([S-])n1-c1cccc(F)c1', 'Clc1cccc([S@@]c2nc3cc(Cl)ccc3[nH]c2=O)c1', 'Clc1cccc([S@@]c2ccc([O-])nn2)c1', 'C=CCNC(=O)C[NH+](Cc1cccs1)[C@@H]1CCS(=O)(=O)C1', 'Oc1ccccc1OS(=O)(=O)c1ccccc1', 'Oc1ccccc1OS(=O)(=O)c1ccccc1', 'Clc1cccc(S(=O)(=O)N(CCSc2nnc(C3CCCC3)n2C2CC2)C2CC2)c1', 'OCc1ccc(-c2csc(-c3ccccn3)n2)cc1', 'N#C[C@@]1(c2cc[nH]c2)CN(CC(F)(F)F)c2ccccc21', 'Clc1ccccc1Oc1cnc(I)cc1O', 'Clc1ccc(Br)cc1-c1cnn2cc(Br)cnc21', 'Clc1ccc(-c2noc(-c3cc(F)ccc3Br)n2)cc1', 'Clc1ccc(-c2nn(-c3cc(Br)cc(F)c3)cn2)cc1', 'Clc1cc(Nc2ncc(I)cc2F)ccc1O', 'O=[N+]([O-])c1cc(OC[C@H]2CCOC2)nc(-c2nccn2C)c1', 'Oc1ccc(/C=N/S(=O)(=O)c2ccc([N+](=O)[O-])cc2)cc1', 'OCc1nnsc1N1CCOCC1', 'N[C@@H](O)c1csc(N2CCCC2)n1', 'ClCc1csc(N2CCOCC2)n1', 'Cc1nnsc1C(=O)N1CCOCC[C@@H]1C', 'CNS(=O)(=O)c1cccc(C(=O)Nc2nc(C3CC3)cc(C)c2[N+](=O)[O-])c1', 'O=C(CNc1nnc(C(F)(F)F)s1)Nc1ccsc1', 'Clc1ccccc1NS(=O)(=O)c1ccc(C(=O)[O-])cc1', 'Clc1ccccc1NS(=O)(=O)c1ccc(C(=O)[O-])cc1', 'O=C([C@H]1[C@H]2CCCC[C@@H]2C1=O)Nc1ccc(I)cc1', 'Clc1ccc(CO)nc1O/N=C/c1c(F)cccc1F', 'Clc1ccsc1-c1cncc(Br)c1', 'Oc1cccc2cc(OC[C@@H](O)[C@@H](c3ccccc3)C(F)(F)F)cnc12', 'COc1nnn(C(=O)Cn2ncc3c(O)cc(Br)c(C)c3c2=O)c1C', 'Clc1cccc(CC[C@H](CO)Nc2ccccc2)c1', 'O=C(Cc1c[nH]c2ccccc12)Nc1ccc(Br)cc1O', 'O=[N+]([O-])c1cc(C(=O)N(Cc2ccco2)C[C@H]2CCOC2)on1', 'Clc1ccccc1-c1csc(SC[C@H](O)C2CC2)n1', 'O=[N+]([O-])c1ccncc1F\n\n', 'O=[N+]([O-])c1ccncc1F\n', 'O=[N+]([O-])c1ccncc1F', 'O=[N+]([O-])c1ccncc1F', 'Oc1ccc([C@H]([O-])Cc2ccccc2)c(F)c1', 'NS(=O)(=O)/N=C(/NCc1cccc(Cl)c1)n1nc(-c2cccs2)c2ccccc21', 'Clc1cc(CCl)n[nH]1', 'Clc1cc(CCl)n[nH]1', 'Clc1cc(CCl)n[n-]1', 'C=CCCOc1ccc(N2C(=O)/C(=C/c3cc(C)on3)C2=O)cc1', 'O=[N+]([O-])c1c(-c2ccsc2)ncc2ccc(OC(F)F)cc12', 'Clc1cc(N2CCC[C@@](O)(C(F)(F)F)C2)ncc1F', 'O=[N+]([O-])c1ccnc2c1NCC2', 'Clc1cc(Br)c(-n2ccnc2)c(Br)c1O', 'C[C@@H](NS(=O)(=O)c1ccc(C(=O)N2CCCC2)cc1)c1cccc(C#N)c1', 'O=C(/N=C/CCC#N)c1ccccc1', 'NS(=O)(=O)N[C@H]1c2ccccc2C[C@@H]1OC(=O)/C(C#N)=C/c1ccc(Cl)cc1', 'Oc1cc(CCNc2nc3ccccc3c(=O)n2-c2ccccc2)ccc1SCC(F)(F)F', 'Clc1ccc(Br)cc1-n1nc(C(F)(F)F)nc1N1CCOCC1', 'COc1ncnc(NCc2cccc(F)c2)c1Br', 'Clc1ccc(Br)cc1-c1cncnc1', 'Clc1ccc(Br)cc1-c1cncnc1', 'Clc1ccc(Br)cc1-c1ccccc1N1CCOCC1', 'O=C(c1cn[nH]n1)N([N+]1CCC1)[C@@H]1CCSc2ccccc21', 'Cc1ncsc1C[NH2+][C@@H](C)c1ccccc1O', 'OCc1n[nH]c(Nc2ccc(Br)cc2)n1', 'ClCc1cc(Br)c[nH]c1=O', 'Clc1cc(F)c(Oc2cncc(C(F)(F)F)c2)cc1', 'Oc1cc(O)cc2[nH]cnc12', 'Oc1cc(O)c2nc[nH]c2c1', 'O=[N+]([O-])c1ccc2[nH]c(C[NH2+]Cc3ccnc(F)c3)nc2c1', 'Oc1ccc(Oc2cnccc2C2CC2)cc1F', 'C=CCOCCCNC(=O)CNC(=O)c1cccc(C(F)(F)F)c1', 'COCc1nnc(NC(=O)CCc2c(C)cccc2C)s1', 'Nc1[nH]nc(Nc2ccccc2F)c1[N+](=O)[O-]', 'CN[C@H]1c2ccc(OC)c(C)c2C(=O)N(CC(C)C)C1', 'Oc1cc(Br)c2s/c(=C\\c3ncc(C(F)(F)F)o3)c(=O)c3ccccc3c2c1O', 'Clc1ccc(Br)cc1-c1cc2c(cc1Cl)NC[C@@H](c1ccc(F)cc1)N2', 'COC[C@H](C)NC(=O)[C@@H]1C[C@H]1c1cc(Cl)ccc1Cl', 'C[C@@H](CCc1ccc(Cl)cc1)c1nnc(NC(=O)C(=O)NC2CC2)s1', 'Clc1ccc(C(CCCO)CBr)cc1', 'Clc1ccccc1Cc1ncoc1-c1cc(F)cc(F)c1', 'Nc1c(Cl)cc(CNC(=O)c2scnc2C2CC2)cn1', 'O=C(/C([O-])=N/S(=O)(=O)c1ccc(Br)cc1)Nc1ccccc1', 'Clc1ccc(-c2cccnc2Cl)c(-n2ccnc2)n1', 'O=[N+]([O-])c1cccc2c1C[C@H]([NH2+]Cc1ccccc1Cl)C2', 'OC1(S(=O)(=O)c2ccc3c(c2)C=CS[C@@H]3c2ccccc2)CCOCC1', 'Clc1ccccc1-c1noc2c1N=C(SCc1ccccc1)C(=O)C[C@@H]2c1cccc(F)c1', 'N[C@@H](CCSc1ccncc1)c1ccc(Cc2ccccc2)s1', 'Oc1ccc([N+](=O)CN2CCN(c3ncccn3)CC2)cc1Cl', 'CN[C@H](C)c1ccc(Cl)cc1', 'Clc1ccccc1OCc1cc(F)cc2ccc(F)cc12', 'Clc1ccc(Br)cc1OCc1cccc(F)c1', 'CC(C)n1nccc1NC(=O)c1cccc(Cl)c1', 'C=C1NC(=O)N([C@@H](C)c2ccccc2C)C(=O)[C@H]1/C=C/c1ccc2c(c1)OCO2', 'Nc1nonc1Sc1ccccc1', 'Nc1cnnn1Sc1ccccc1', 'Oc1ccc([N+](=O)[O-])c(/C(=N/c2ccc(Cl)cc2)[n+]2ccccc2Cl)c1', 'O=C([C@H]1c2cc([N+](=O)[O-])ccc2Sc2ccccc21)[NH+]1CCN(c2cccc[nH+]2)CC1', 'Cc1snnc1CN(C)C(=O)c1ccc([S@](C)=O)cc1', 'Oc1cc(-n2nnc(C(F)(F)F)n2)ccc1-c1nc2ccccc2s1', 'Oc1cccc(CCNc2c(Cl)cc(Cl)cn2)c1', 'OC[C@@]1(c2ccccc2)Nc2sccc2N1', 'O=C(c1ccsc1)[C@@H]1COc2ccccc2O1', 'Clc1cc(N2CCNC[C@@H]2c2cccc3ccccc23)ccn1', 'O=C(c1ccc[nH]c1=O)N1CCN(c2nccc(Br)n2)CC1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1N[C@H]1CCCS1', 'COc1ccc(C[C@@]2(C)NC(=O)N(CC(=O)N(C)S(=O)(=O)Cc3ccco3)C2=O)cc1', 'Clc1cc(Cl)nn1C[NH+]1CCN(c2ccccc2)CC1', 'ClCc1nnc(Oc2cccc([N+](=O)[O-])c2)n1C1CC1', 'NS(=O)(=O)C[C@@H](SCC(=O)c1ccsc1)C(=O)[O-]', 'NS(=O)(=O)CCc1nc(-c2ccccn2)no1', 'COc1ncccc1COC(=O)c1cccc2cc[nH]c12', 'O=C(c1cc(F)ccc1C(=O)c1ccc(Cl)s1)N1CCCCC1', 'COc1ncnc(N(C)[C@@H]2CCO[C@]3(CCSC3)C2)c1C', 'Clc1cccc(/N=C(\\[O-])NCc2ccco2)c1O', 'O=C(CS(=O)(=O)c1ccc(Br)cc1)Nc1cc(Br)on1', 'N#CCS\\C(=N\\OCCc1cccc(Cl)c1)c1ccccc1', 'Clc1ccccc1Oc1cc(Cl)c([N+](=O)[O-])cc1[N+](=O)[O-]', 'O=[N+]([O-])c1c(Br)c([C@@H]2CCCCO2)cs1', 'O=C(/C=C/O)C(=O)NCc1ccccc1Br', 'ClCc1cnn(-c2cccc(Cl)c2)c1Cl', 'OCc1n[nH]c([C@@H](C#N)c2ccc(Cl)cc2)c1Cl', 'Clc1cc(Cl)cnc1C[NH+]1CCc2oncc2C1', 'Clc1ccccc1-c1cscc1-c1cc2c(cc1Cl)OCCO2', 'Clc1cc(/N=C\\CSc2ccc(Cl)cc2Cl)ccn1', 'ClC(=S)NCC(=O)Nc1ccccc1F', 'Clc1nncc(-c2c(F)cccc2F)n1', 'Clc1cccc(F)c1-n1cnnc1', 'O=C(c1ccsc1)N1CCN(Cc2ccccc2)CC1', 'O=C([C@H](Sc1cc(Cl)ccc1Cl)n1cccn1)Nc1n[nH]c(=O)[nH]c1=O', 'C[C@@H](Nc1ncccn1)[C@@H]1CCCN(c2ncccc2C(C#N)=O)C1', 'O=C(CNc1ncon1)N[C@H]([C@@H]1CCS(=O)(=O)C1)c1ccccc1', 'Clc1ccccc1CCNC1=NC=CCN1', 'O=[N+]([O-])c1ccnc2c1[C@@H](N[C@@H]1CCOC1)CS2', 'Clc1cc(/N=C/c2cccs2)ccc1Cl', 'Clc1ccsc1N[C@H]1CCOc2c(F)cccc21', 'Clc1ccc(Br)cc1CC(=O)N/N=C/c1ccc(F)cc1', 'Clc1ccccc1Nc1ncnc2[nH]c(C3CC3)nc12', 'Clc1ccsc1C[N-]c1ccc(C(F)(F)F)nc1', 'C=C[C@@H]1N[C@]2(C(=O)NCCN2c2ccccc2)c2ccccc21', 'Clc1nncc(F)c1C(=O)N1CCCc2ccc(Br)cc21', 'O=[N+]([O-])c1ccn(CC(=O)N2CCSc3ccccc32)c1', 'Clc1cccc(CCn2c(-c3cc(Cl)ncc3[N-]S(=O)(=O)c3ccccc3)no2)c1', 'Clc1cccc2cc(/C=C/Br)ccc12', 'O=C(/C=C\\C=C\\c1ccccc1)Nc1ccc2c(c1)OCO2', 'Clc1ccc(-c2n[nH]cc2C(F)(F)F)cc1', 'Oc1cc(Br)cc(F)c1N1CCN(c2[nH+]cccc2[N+](=O)[O-])CC1', 'OC[C@@]1(O)CCc2ccccc2CN1c1ccc(F)cc1', 'Clc1ccsc1SCc1ccc(Cl)cc1Cl', 'O=[N+]([O-])c1c(Cl)nn(-c2ccccc2)c1Nc1ccc([N+](=O)[O-])cc1', 'Clc1ccccc1-c1n[nH]cc1-c1ccc(Cl)cc1Cl', 'Clc1cc(Br)c([C@H](O)c2ccccn2)cc1F', 'O=C(c1ccsc1CN1C(=O)CCCC1=O)Nc1cc(C(F)(F)F)ccc1F', 'Nc1[nH]nc2ncccc12', 'Nc1[nH]nc2ncccc12', 'Clc1ccccc1C(=O)N/N=C\\c1ccccc1O', 'Clc1ccc(C(=O)Oc2ccccc2C(=O)[O-])cc1', 'ClCc1nnc(COc2ncccc2F)s1', 'Oc1ccc(/C=C/NC(=O)CSc2nc3ccccc3nc2N)c(Cl)c1', 'Clc1nnnn1CCc1ccccc1', 'Clc1ccc(Br)cc1NCC[C@@H]1CCCC[NH2+]1', 'ClCc1csc(S[C@H]2CCN(Cc3ccccc3)C2)n1', 'Clc1cccc2cccnc1n2-c1ccccc1', 'Clc1ccc(-c2nnnn2-c2cc(C(F)(F)F)cs2)cc1', 'Oc1cc(C=O)ccc1OCC(F)(F)F', 'N=C1/C(=C\\C=C\\c2cccc(Cl)c2)C[NH2+]C1', 'NS(=O)(=O)C[C@H](SCc1ncoc1C1CC1)c1ccccc1', 'Oc1ccc([N+](=O)[O-])c(I)n1', 'Oc1ccc([N+](=O)[O-])c(I)n1', 'Oc1ccc([N+](=O)[O-])c(Br)n1', 'O=C(c1c[nH]cn1)[C@H]1Oc2ccccc2NC1=O', 'Nc1cn[nH]c(=O)c1N', 'ClC(c1ccc(F)cc1)[C@@H](O)c1cc(Br)cc(F)c1', 'O=C(CSCc1noc(=O)o1)Nc1ccc(F)cc1Cl', 'Clc1cc(NSc2ccc(Cc3nc[nH]n3)o2)ccc1OCc1ccc(Cl)cc1', 'O=[N+]([O-])c1cc(Cl)c(Cl)cc1Oc1ccc2ccccc2c1', 'COC(CNC(=O)N1CC[C@H](C)[C@@H](O)C1)c1cccc(F)c1', 'O=C([C@H](Cl)Br)c1cccs1', 'Oc1cc(OCC(F)F)ccc1Oc1cccc(Cl)c1', 'Oc1cc(C[NH2+][C@@H]2CCc3ccccc32)ccc1Br', 'Clc1nnc(Cl)[nH]c1=S', 'Oc1cccc(Br)c1Oc1ccc(Br)cc1', 'O=C(CNS(=O)(=O)/C=C\\c1ccc(F)cc1)Nc1ccc2c(c1)OCO2', 'Clc1ccccc1Oc1cnc2cc(C(F)(F)F)c[nH]c12', 'O=[N+]([O-])c1c(Cl)n2nnnc2n1Cc1ccc(F)cc1', 'O=[N+]([O-])c1c(Cl)n2nnnc2n1Cc1ccc(F)cc1', 'Nc1nc2c(n1Cc1ccc(C(F)(F)F)cc1)CCC2', 'Oc1ccccc1-n1nnnc1SCC(=O)N1CCc2ccccc21', 'Cc1sc2ccccc2c1CC(=O)NC1CCC(CO)CC1', 'O=C([C@H](Cl)Br)[C@@H](C)c1ccc(Cl)cc1Cl', 'O=C(c1cnn(Cc2ccccc2C#N)c1)N1CCCCC1', 'Nc1[nH]nc(SCC(=O)Nc2nc3ccccc3[nH]2)c1Cl', 'Clc1ccccc1Nc1nc2ccccc2cc1CO', 'Clc1nnc(CSc2c[nH]c3ccccc23)[nH]1', 'Clc1ccc(-c2cc(F)c3[nH]ccc3c2)cc1F', 'Clc1cc(COC2=N[C@@H]3CS(=O)(=O)C[C@@H]3N(c3ccncc3)C2)ccc1Cl', 'COc1nnnc(Nc2c(Cl)cccc2Cl)c1C', 'ClC[C@H]1Oc2ccccc2NC1', 'ClC[C@@H]1Oc2ccccc2NC1', 'Clc1ccsc1/CC(=O)N/N=C/c1ccc2cc(Cl)ccc2n1', 'Clc1ccccc1-c1cscc1C(F)(F)F', 'Clc1ccccc1-c1cscc1C(F)(F)F', 'Clc1ccccc1-c1cscc1C(F)(F)F', 'Oc1cccc2ncn(/c3c(C(=O)NCCc4ccccc4)c3ccccc2)c1', 'Nc1noc2cc(CCSc3ccc(O)cc3)ccc12', 'Oc1ccc(Oc2nc[nH]n2)c2ncccc12', 'Clc1cccc([S@@]c2n[nH]c(=O)n2C[C@H]2CCCO2)c1', 'Cc1sc(C[NH2+][C@@H]2CCCC[C@@H]2CO)c(C)c1C', 'Clc1cccc(NC[C@]2(O)CCNC2=O)c1', 'NS(=O)(=O)Cc1nnc2ccc(Br)cn12', 'Clc1ccc(Br)c(SCc2nnc3n2CCC3)c1', 'Clc1nnnn1c1c(C(F)(F)F)ccc2c1OCO2', 'Clc1ccc(-c2nonc2)nc1C(F)(F)F', 'Clc1ccccc1Oc1cnc2cc(F)c(F)cc2c1[O-]', 'Clc1ccc(-c2nn(-c3ccc(Cl)cc3Cl)cn2)cc1', 'O=C(CCc1cc[nH+]cc1)Nc1cccc(CNc2ncccn2)c1', 'Clc1cc(/C=N\\Oc2cccc(Br)c2)cc([O-])c1Cl', 'O=C(CSCc1c(N)cnn1Cc1ccccc1F)N/N=C/c1ccc2c(c1)OCO2', 'Clc1ccccc1N1Nc2[nH+]ccnc2NC1', 'Clc1cccc([C@@H](O)CNc2cccc(F)c2)c1', 'Clc1ccccc1Oc1cc(Br)c(I)c(F)c1', 'COc1nn(-c2c(F)cccc2Cl)c2c1[C@@H](c1ccc(OC)cc1C)C2', 'Clc1ccccc1NS(=O)(=O)c1c[nH]ccc1=O', 'Clc1ccccc1NS(=O)(=O)c1c[nH]c(=O)[nH]c1=O', 'OC[C@@]1(Nc2cccc([N+](=O)[O-])n2)CCCc2ccccc21', 'O=C(c1cn(Cc2ccco2)c2c1CCCO2)N1CCSCC1', 'Oc1cccc2cc(C(=O)OCCN3CCOCC3)cnc12', 'CN(c1ccn([C@@H]2CCS(=O)(=O)C2)c1)c1nc(C2CC2)no1', 'Clc1ccccc1-c1noc(Cc2cccs2)n1', 'ClCc1nonc1-c1cc(Br)cc(Br)c1', 'Clc1ccccc1C(=O)[O-]', 'CNc1ccc(F)c(C[NH+]2CCCC2)c1', 'Oc1ccc(F)c(CCN2CC(=O)Nc3ncccc23)c1', 'NS(=O)(=O)N[C@@H](c1ccccc1)c1csc2ccccc12', 'Clc1ccccc1Nc1nncc([O-])c1[N+](=O)[O-]', 'Cc1sncc1CNc1ccnc(OCC2CC2)c1', 'Clc1ccc(Cc2ncco2)cc1O', 'Clc1cc(Br)c(F)c(/N=C2/S[C@@H]3CSC[C@@H]23)c1', 'Clc1cc(Cl)nc2ccc(C(=O)OC[C@@H]3CCCO3)cc12', 'Oc1cccc2cc(O[C@H](Cc3ccccn3)[NH+]3CCCC3)nnc12', 'Clc1ccc(Cc2ccccc2)c(F)c1', 'Oc1ccc(F)cc1-c1onc(N2CCCCCC2)n1', 'Clc1ccc(-c2cc(/C=C3\\C(=O)N(Cc4ccco4)C3=O)oc2[O-])cc1', 'O=C(c1c[nH]c2cc(Cl)ccc12)c1ccc(Br)cc1', 'Nc1nonc1C=O', 'N=Cc1nonc1N', 'Clc1cc(NSc2nnc(COc3ccccc3Cl)[nH]2)cc(Cl)c1Cl', 'Clc1cc(Br)cc([C@@H]2CCCN2C(=O)c2cccnc2)n1', 'Clc1ccsc1-c1nnnn1-c1ccc(F)c(Cl)c1', 'N#C[C@H](c1ccccc1)[C@H]1CCC[NH+](Cc2ccc(C(C)(C)C)o2)C1', 'Oc1ccc(/C=C2\\CCS(=O)(=O)C2)c(Br)c1', 'Clc1cccc(S[C@@H]2OCCNC2=O)c1', 'Oc1ccc([N+](=O)[O-])c(/C=C2\\S[C@@H]3CS(=O)(=O)C[C@@H]3N2C(=O)c2ccco2)c1', 'Clc1ccc(C(=O)N2CCCC2)c2c1cnn2-c1ccc(Cl)cc1Cl', 'O=C(c1ccsc1)[C@@H]1CS[C@@H]2[C@H](C=C3C=CC=CC3=N2)C1', 'Clc1cc(N2CCOCC2)nc(-c2ccco2)n1', 'O=[N+]([O-])c1c(Cl)ncc2cc(S(=O)(=O)Nc3ccc(Br)cc3F)ccc12', 'Clc1ccccc1Oc1cncc(-c2ccc(F)cc2Cl)n1', 'Clc1ccccc1-c1noc(/C=C/c2ccc(F)cc2Cl)n1', 'Clc1cc(Nc2ccccc2Cl)nc(-c2ccccn2)n1', 'Clc1cc(Br)c(NCc2ccncc2)nc1Cl', 'Oc1ccc([N+](=O)[O-])c(S(=O)(=O)NCc2c(O)cccc2O)c1', 'Oc1cccc(-c2cncc(Cl)n2)c1O', 'Oc1ccc([N+](=O)c2ccc(O)cc2)cc1OCc1ccccc1', 'Clc1ccc(Br)cc1C[NH2+][C@@H]1CCC[C@H]1O', 'O=[N+]([O-])c1ccncc1C[NH+]1CCN[C@H](c2ccc(Cl)c(Cl)c2)C1', 'Oc1ccc([N+](=O)c2occc2Cl)c2c1OCCO2', 'Cc1c(Br)oc2nc(CCCl)ncc12', 'N#C[C@@H]1CNCCN1Cc1cc2c(cc1F)OCO2', 'O=C(c1ccsc1Cl)Oc1cccc2c[nH]cc12', 'Clc1ccc(Br)cc1O[C@@H]([NH3+])C1CCCC1', 'Clc1ccc(Br)c([C@@H]2CCCN(S(=O)(=O)c3cc(F)cc(F)c3)C2)c1', 'Clc1cc(F)ccc1-c1cccc(S(=O)(=O)/N=C(\\[O-])CCC[C@@H]2CCCO2)c1', 'ClC[C@H]1OCc2ccccc21', 'Clc1ccccc1OCc1ccc(Cl)cc1Cl', 'Clc1ccccc1OCc1cc(F)ccc1F', 'Clc1ccccc1OCc1cc(Cl)ccc1F', 'Clc1ccc(-c2cc([C@H]3CCCO3)n[nH]2)cc1', 'Clc1ccccc1NS(=O)(=O)c1ccccc1Br', 'Clc1ccccc1NS(=O)(=O)c1ccc(Br)cc1', 'Oc1cccc(CC[NH2+]Cc2cccc(C(F)(F)F)c2)c1', 'O=C(c1cc([N+](=O)[O-])ccc1Cl)N1CCN(Cc2cc(C3CC3)cs2)CC1', 'Oc1ccc(F)cc1C[C@@H]([NH3+])c1ccccc1F', 'Clc1cc(/N=C/CSc2cc(Br)c3ccccc3n2)ccc1O', 'Oc1cc(Br)cc(/C=C2\\CCC([C@H]3C=CC=N3)N(c3ccc(Cl)cc3)C2=O)c1O', 'O=[N+]([O-])c1c(Cl)nc2ncon12', 'Oc1ccc(F)cc1[C@H]1CCCN(c2nccs2)C1=O', 'O=C(c1cccc(-n2cccc2)c1)N1CCC(Oc2cccnc2Cl)CC1', 'O[C@@H]([C@@H]1CSc2ccc(F)cc21)c1ccc(Br)cc1', 'N=Cc1nonc1[N-]S(=O)(=O)c1ccc2c(c1)OCCO2', 'NS(=O)(=O)Cc1nnc(-c2ccccc2)n1-c1ccc(Cl)c(Cl)c1', 'Oc1cccc2nc(SCCSc3nc4ccccc4cc3n1)CCC2', 'Oc1cc(-n2nnnc2SCc2ccccc2)ccc1OCc1cccnc1', 'ClC(c1ccccc1)c1cccc(Cn2cncn2)c1', 'CNS(=O)(=O)Cc1ccc(F)c(C(=O)Nc2cccc(F)c2)c1OC', 'O=[N+]([O-])c1ccnc2cccnc12', 'O=[N+]([O-])c1ccnc2cccnc12', 'O=[N+]([O-])c1ccnc2cccnc12', 'O=[N+]([O-])c1ccnc2cccnc12', 'O=[N+]([O-])c1c(Br)cnc2cccnc12', 'N#Cc1cccc(S(=O)(=O)NC2CC2)c1', 'Clc1ccsc1/N=C/c1cccc(COc2cccnc2)c1', 'Clc1cc(Cl)ncc1[O-]', 'ClC[C@@H]1C[C@@H]1c1cccc(Br)c1', 'NS(=O)(=O)Cc1ccsc1', 'O=[N+]([O-])c1ccncc1SC[C@@H]1C[C@H](O)c2ccccc2O1', 'O=[N+]([O-])c1c(F)cc2n1CN1CCCC[C@H]1CO2', 'Cc1c(Br)cccc1N[C@H](C)C(=O)NNC(=O)c1cccs1', 'Clc1ccc(CCBr)c(CN2CC[NH2+]CC2)c1', 'Oc1cc(C[NH2+][C@H]2CCOC3(OCCCC3)C2)ccn1', 'Clc1ccc(-c2ccccc2[N-]C2=NC([N+](=O)[O-])C3=C2C(=O)CCC3)cc1', 'Clc1ccc(-c2cccnc2N2CCC[C@@H]2C(F)(F)F)cc1', 'Clc1cccc(Nc2ncnc(Nc3ccccc3)c2[N+](=O)[O-])c1', 'Clc1cccc(NCCc2nccc(-c3cccc([N+](=O)[O-])c3)n2)c1', 'Clc1ccc(COc2ncc3cccc(Br)c3c2)cc1', 'Clc1ccc(Br)cc1S(=O)(F)c1cnn(Cc2ccccc2)c1', 'Clc1ccc(C2CN2)cc1', 'Oc1ccc(/C=C(/CO)C(F)(F)F)cc1', 'O=C(c1cccnc1Cl)N1CCC(CCNc2cccc(Cl)c2)CC1', 'Clc1ccccc1CS(=O)(=O)c1ccccc1Cl', 'Clc1ccc(Br)cc1[C@@H](Cl)S(=O)(=O)c1ccccc1', 'O=C([O-])C[C@H]1CCC[NH+](C[C@@H](O)c2cc(Cl)ccc2F)C1', 'Clc1cc(F)c(NC2CC[NH+](CCCC(=O)[O-])CC2)cc1', 'N#CC(/N=C/c1ccc([N+](=O)[O-])cc1)=C/C(=O)Nc1nccs1', 'O=C(c1ccc(Cl)cc1)N1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1', 'Clc1cccc(Nc2cc(S(=O)(=O)N3CCCCC3)ccc2Cl)c1', 'CNc1cccc(CNC(=O)Nc2ccc(F)c(Cl)c2)c1O', 'Oc1cc(-n2nc3sc4ccccc4nc32)cs1', 'O=C([O-])[C@@H]1CO[C@H](c2ccccc2)S1', 'O[C@@H](c1cscc1C(F)(F)F)C1CC1', 'OC1(c2ccccc2)CCN(C(=O)c2csc(=O)[nH]2)CC1', 'O=C(CS(=O)(=O)[N-]C(=O)CCCc1cccc(Cl)c1)Nc1ccccc1', 'Clc1ccc(Br)cc1/C=N/N=C1\\S[C@@H]2CS(=O)(=O)C[C@H]12', 'Clc1cc(F)c(OCC(F)F)cc1I', 'Clc1cc(NSc2ccn[nH]2)ccc1N/N=C/Oc1cccnc1', 'O=C(CNc1c(F)cccc1F)Nc1ccc(Oc2ncccn2)cc1', 'O=C([O-])/C(=N/c1ccccc1)c1ccc(Br)o1', 'Clc1cccc(Br)c1N[C@@H]1CCc2ccccc21', 'Clc1cc(F)c([O-])c(C[NH+]2C[C@@H](CO)C2)c1', 'ClCc1ccnn1Cc1ccc(F)cc1O', 'Clc1ccc(Br)cc1S[C@H]1CCC(=O)NC1=O', 'Clc1ccc(-c2cc(Br)cs2)cc1', 'Clc1cccc2[nH]nc(NC(=O)Cc3cccnc3)c12', 'Clc1ccc(Br)cc1OC[C@H]1CC=CCC1', 'Clc1ccc(Br)cc1OCC1CCCCC1', 'NS(=O)(=O)C[C@@H](Br)c1ccccc1', 'Clc1cccc(S[C@@H]2CCN(c3ccccn3)C2)c1', 'CCCCCNC(=O)c1cc(NC(=O)C=C2CCCCC2)co1', 'Clc1cccc([S@@]c2nnc(-c3ccccc3Br)n2C2CC2)c1', 'Clc1ccccc1OCCSc1nc(-c2ccco2)n[nH]1', 'ClCc1cc(CNc2cccc(C(F)(F)F)c2)n[nH]1', 'Clc1cccc(S(=O)(=O)Oc2ccccc2Br)c1', 'Oc1ccc(F)c(-c2[nH+]c3ccc(Cl)cc3o2)c1', 'Clc1ccccc1Oc1cnc2cc(Br)ccc2n1', 'Clc1ccccc1Oc1cnc(Cl)c(Cl)c1', 'Clc1ccccc1Nc1cnc(F)cc1Cl', 'Clc1ccc(Br)cc1-c1cn(-c2ccccc2)nn1', 'Clc1cc(Cl)nnc1Oc1ccc(Br)cc1', 'Clc1cc(Cl)nnc1Nc1ccc(Br)cc1', 'Clc1cc(Cl)nnc1-c1ccccc1Cl', 'Clc1cc(Cl)nc(Nc2ccc(F)cc2)c1', 'O=[N+]([O-])c1c(Cl)nnc2c1COC2', 'O=[N+]([O-])c1cccc2[nH]cnc12', 'O=C(CS(=O)(=O)Cc1cccc(F)c1)Nc1ccccc1OCc1ccccc1', 'Clc1ccc(Br)cc1NC1=CSCCS1', 'Clc1ccsc1[C@H](CSc1nnc(Cc2ccccc2)n1C(F)F)N1CCCC1', 'O=C(c1c[nH]nc1-c1ccc2c(c1)OCO2)N(C1CCCCC1)S(=O)(=O)c1ccccc1', 'Clc1cc(Nc2c([O-])n3ccccc3n2)ccc1Br', 'O=C(/N=C/c1cc2ccccc2[nH]1)C1(c2cccnc2)CC1', 'O=C(/N=c1/[nH]c2ccccc2cc1C(F)(F)F)C(=O)N1CCC[C@@H](c2nnc[nH]2)C1', 'O=C(/N=c1cc2oc3ccccc3c(=O)n1[C@@H]2c1cccs1)c1ccc(Cl)cc1', 'Oc1cc(-n2cnc(CN3CCOCC3)n2)ccc1F', 'Oc1ccc(O[C@H]2CC[NH2+]C[C@@H]2CC2CC2)c(Cl)c1', 'Clc1ccc(-c2csc3[nH]c(N[C@H]4CCO[C@@H](CO)C4)nc23)cc1', 'N#C/C(=C1\\N=C[C@H](S/C=N/S(=O)(=O)c2ccccc2)S1)c1ccccc1', 'Clc1ccccc1NS[C@H]1CCCN(CCc2cccnc2)C1', 'Clc1ccc(-c2nonc2N2CCC[NH2+]CC2)cc1Cl', 'Clc1ccc(-c2nonc2NC[C@@H]2CCCSC2)c(F)c1', 'O=C(C[NH+]1CCN(c2cccc(F)c2)CC1)Nc1ncc(-c2nccn2C(F)F)cn1', 'O=C(CNC(=O)C1(c2ccccn2)CCOCC1)Nc1cnn(Cc2cc(F)ccc2)c1', 'NS(=O)(=O)Nc1cc([S@@](=O)Cn2cccn2)ccc1C(F)(F)F', 'O=C([C@H](C#N)c1ccc(Cl)cc1)N1CCN(c2ccc(Cl)cn2)CC1', 'O=C(CS(=O)(=O)[C@H](Cc1ccccc1)N1C(=O)c2ccccc2C1=O)c1ccco1', 'N#C[C@H](c1ccccc1)[C@@H]1CCCN1Cc1ccc(F)c(F)c1', 'Oc1ccc(Oc2nc(=O)c3cccnc3n2N)c([N+](=O)[O-])c1Cl', 'Cc1sn(-c2ccccc2)c(=O)c1NC(=O)c1cc[n+]([O-])cc1', 'Clc1ccc(Br)cc1/N=N/SC1=NC=N/c2nccnc21', 'NS(=O)(=O)/C(=C\\c1ccc(Cl)cc1)c1[nH]c2ccccc2c1Oc1ccc2c(c1)OCO2', 'Clc1ccc(Br)c([C@@H](O)c2cccnc2)c1', 'Clc1cc(CSc2nncn2-c2ccc(Br)cc2)c2ccc(F)cc2c1O', 'Clc1nnn(-c2cnn(Cc3cccnc3)c2Cl)c1Cl', 'O=C([C@H](CNC(=O)CSCc1ccc(Cl)cc1)N1CCCC1)Nc1ccccc1Cl', 'O=C(c1cccs1)c1cnn(-c2ccc(F)cc2)c1', 'Clc1cc(CSC2=NC[C@H]3CS(=O)(O)C[C@H](O)[C@H]23)ccc1O', 'Clc1nnnn1/N=C/C(=O)c1ccc(F)cc1', 'O=C(C[NH+]1CCOCC1)NCCCc1ccc(F)cc1Cl', 'ClCc1cc(Cl)cc(C(=O)NCC[NH+]2CCOCC2)c1', 'O=[N+]([O-])c1c([O-])c2nc(-c3ccccc3)n[nH]c2c1O', 'Clc1ccccc1N1N=C/c2ccccc2[C@H]1c1ccccc1F', 'N[C@@H](c1ccc(F)cc1)c1c(F)cccc1Cl', 'Oc1ccc(F)cc1SC1CCN(C(=O)c2ccc(Cl)nc2)CC1', 'Clc1cccc(F)c1[C@@H]1Oc2c(cccc2Cl)C1', 'Clc1ccccc1Nc1cnc([S-])nc1', 'O=C(c1cccc(Br)c1)N1CCN(c2nccc([N+](=O)[O-])c2)CC1', 'Nc1nonc1NCC(=O)Nc1cc(Cl)ccc1Cl', 'O=C(c1ccsc1)N(Cc1cccs1)[C@@H]1CCS(=O)(=O)C1', 'Clc1ccccc1-c1nnc(CSc2ncccn2)o1', 'Clc1cc(Cl)nnc1O/C=C(/CC#N)c1ccc(Cl)cc1', 'CN(C)c1ncccc1C(=O)Nc1ccc(N2CC[NH+](CC(Cl)(Cl)Cl)CC2)cc1', 'Clc1cccc(N2N=C(CC3=c4ccccc4=[NH+]C3)CC2)c1', 'O=C(Cc1n[nH]n2c[nH+]cc12)Nc1cccnc1', 'O=C(CS(=O)(=O)[C@@H]1CCSC1)Nc1cc(C(F)(F)F)cs1', 'O=C([C@H](C(=O)[O-])c1ccc(Cl)cc1)c1ccc(F)cc1', 'O=[N+]([O-])c1cc(Cl)nc([N-]SCc2ccc(OCc3ccccc3)cc2)c1', 'O=[N+]([O-])c1cc(SCc2ccco2)nc(Oc2ccccc2)c1', 'NS(=O)(=O)/C(=C(\\C)c1csc(NC(=O)c2ccc(F)cc2)n1)c1ccccc1', 'Clc1cccc(/C=N/Nc2c(Cl)cccc2Cl)c1', 'Clc1nnnn1c1cccc(OCCO)c1', 'Cc1sncc1C(=O)O[C@@H]1[C@@H](O)[C@H](C)OC[C@@H]1OCc1ccccc1', 'Clc1ccccc1O/C=Cc1ccc(N2S(=O)([O-])N=C(O)COc3ccccc32)cc1', 'ClC(/N=C1/SC2=C(CCCC2)SC1=S)c1ccccc1', 'O=C([O-])[C@@H]1C[C@H]1c1cccc(Cl)c1Cl', 'O=C(/N=C\\c1c[nH]c2ccccc12)c1ccc([N+](=O)[O-])cc1', 'Oc1ccc([N+](=O)[O-])c(CN2C[C@@H](O)C[C@H]2c2ccccc2)c1', 'O=C(c1cc(Cl)c2nccnc2n1)N1CCC[C@H]1c1ccc(F)cc1', 'O=C(c1cc(Cl)c2[nH]ccc2c1)N1CCN(c2cnccn2)CC1', 'O=C(c1c[nH]c([N+](=O)[O-])c1)Oc1cc2c(cc1Cl)OCCC[C@@H]2O', 'Clc1ccc(Cc2nnn(-c3ccccc3)n2)cc1Cl', 'Clc1ccc(Br)c(OCc2nnc3ccccn23)c1', 'Clc1cc(/N=C/c2noc(-c3ccccc3)n2)cc(Cl)c1', 'Clc1nncn1c1ccc(Cl)c(Cl)c1', 'Clc1nncn1c1cc(Cl)ccc1Cl', 'Clc1nncc(-n2cccc2)c1-c1cc(F)ccc1F', 'Clc1ccc(Br)cc1-n1cnnc1', 'Clc1ccc(Br)cc1-n1cncn1', 'Clc1ccc(Br)cc1-n1cn[nH]c1=O', 'Clc1ccc(Br)cc1-n1ccnc1CO', 'Clc1nnnn1-c1cnccc1', 'Clc1nnnn1-c1cnccc1', 'Clc1cc(/N=C\\N2CCCCCCC2)ccc1[N+](=O)[O-]', 'Clc1cccc2[nH]nnc1c1cc(C3CC3)nn12', 'Clc1cccc(NC2[C@H](Cc3ccccc3)O2)c1', 'C[C@@H]([C@@H](O)COc1ccccc1)N(C)c1ccccc1', 'Nc1nnnn1CC[S@@](=O)c1ccccc1Cl', 'O=C([C@H](Sc1ccsc1)c1ccc(F)cc1)N1CCc2ccccc21', 'Clc1ccc(Br)c(-n2nnnc2SCc2csc(-c3ccccc3)n2)c1', 'Clc1ccsc1-c1nc([C@@H]2C[NH2+]CCO2)no1', 'Clc1cc(/N=C\\c2ccccc2)cc2c1CCCC2', 'Oc1cccc(C[NH2+]Cc2ccc(Oc3ccccc3)cc2)c1', 'O=C([C@H]1COCCN1S(=O)(=O)c1ccccc1Cl)N1CCC(c2nccs2)CC1', 'Clc1cccc2[nH]nc(-c3nnc(-c4cccc(Cl)c4F)[nH]c13)CC2', 'Oc1ccc(F)c(-c2nnc(NC[C@@H]3CCCO3)s2)c1', 'Clc1cc(/C=C(/Br)C(F)(F)F)ccc1Cl', 'Clc1ccsc1NC(=S)NN=C1CCc2ccccc21', 'Cc1snnc1S(=O)(=O)Cc1ccc([N+](=O)[O-])cc1', 'Clc1ccc(Br)cc1Cc1nnnn1C1CC1', 'Clc1ccc(-c2nnnn2Cl)cc1[N+](=O)[O-]', 'Oc1ccc([N+](=O)c2oncc2C2CC2)cc1F', 'COc1ncncc1C(=O)Nc1ccc([N+](=O)[O-])cc1', 'Clc1cccc(/N=N/NNNc2ccccc2)c1', 'Oc1cc(Br)cn/c(=[N+]([O-])c2ccccc2)c1', 'O=C(CNS(=O)(=O)c1ccccc1)Nc1cccc(-c2nc[nH]n2)c1', 'Nc1n[nH]c(-c2cccc(F)c2)c1NC(=O)c1ccc(S(=O)(=O)C(F)F)cc1', 'Clc1ccccc1NS[C@H]1Oc2cc(Cl)ccc2S1', 'Clc1ccc(-c2cccs2)[nH]1', 'Clc1ccc(-c2cccs2)[nH]1', 'Clc1ccc(-c2cccs2)[nH]1', 'Clc1nncc(O)c1', 'Clc1ccc(Br)cc1OS(=O)(=O)C1CC1', 'Clc1cccc(Br)c1Oc1cccc2c1CCC2', 'Clc1nncc(S(=O)(=O)c2ccc(Br)cc2)n1', 'O=C(Cc1cnc2ccccc2c1)Nn1ncc(Cl)c1C(F)(F)F', 'Oc1cc(-n2cc(-c3ccccc3)nn2)ccc1Br', 'Cc1cnc2[nH]c(C(=O)Nc3ccc4c(c3)OCO4)cc2c1', 'O=C(CNc1nn(Cc2cc(F)cc(Cl)c2)c(=O)c2occc12)Nc1cccnc1Cl', 'ClCc1nnsc1NC1CCCCCC1', 'O=C(CNS(=O)(=O)c1ccc(F)cc1)N[C@H]1CCSc2ccccc21', 'Oc1cc(-n2cc(I)c([O-])n2)ccc1F', 'Cc1sc2nc(NC(=O)C3=CN(c4ccccc4)CC3)sc2c1C', 'Clc1ccccc1Oc1cc(SCc2csc(C(F)(F)F)n2)[nH]n1', 'O=C(c1cccnc1)N1CCC(Nc2ccccn2)CC1', 'O=C(CNS(=O)(=O)Cc1ccccc1)Nc1ccc(F)c(F)c1F', 'Clc1ccc([C@@H]([O-])Cn2cncn2)c(Cl)c1', 'Clc1ccc(-c2csc3sc(-c4ccoc4)cc23)cc1O', 'Clc1ccccc1-c1csc([C@@H](OCC(F)(F)F)c2cc(F)cc(F)c2F)n1', 'ClCc1csc(N2CCNC2=O)n1', 'Clc1ccccc1[C@H]1C[C@@H](c2cccc(F)c2)CS1', 'O=C(CSCc1ncc(Cl)cn1)NC[C@@H]1CCc2ccccc2O1', 'Oc1cccc(C[NH+]2CCC[C@H](Nc3ccc(Cl)cc3)C2)c1', 'O=C(c1ccccc1F)N1CCC[C@@H](n2nccc2CS(=O)(=O)C(F)(F)F)C1', 'Clc1ccccc1-c1csc(S(=O)(=O)Cc2ccccc2)n1', 'NS(=O)(=O)Cc1nc2cccc(Cl)c2[nH]1', 'Oc1cc(Cn2c[nH+]c(-c3ccccc3)n2)ccc1Cl', 'Clc1ccc(-c2cc[nH]n2)cc1Cn1ccccc1=O', 'Clc1ccc(-c2cc(/C=N/c3ccccc3)[nH]n2)o1', 'Clc1cc(Cl)nc(COc2ccccc2F)n1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1-c1ccc(F)cc1', 'O=C(/C(CNc1ccccc1Cl)c1ccc([N+](=O)[O-])cc1)c1ccccc1F', 'Nc1n[nH]c(-c2cccc(Br)c2)c1[O-]', 'Clc1ccc(Br)c(F)c1-c1cc([N+](=O)[O-])cnc1-c1ccccc1', 'Clc1cc(Br)cnc1OCc1cc(F)ccc1Br', 'Clc1cc(Br)cnc1COc1ccc(Cl)cc1Cl', 'Clc1ccc(-c2csc3[nH]nnc23)cc1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1', 'Clc1ccc(-c2csc3[nH]ncc3c2=O)cc1', 'Clc1ccc(-c2csc3[nH]ccc23)nc1O', 'NS(=O)(=O)CC[C@@H]1N=Nc2ccccc21', 'NS(=O)(=O)/C=c1/c(=N/C(=O)c2cc(Cc3ccccc3)n[nH]2)sc2c1CCCCCCC2', 'Clc1cc(F)c(O[C@@H]2CCC[C@@H](C[NH3+])C2)cc1', 'Clc1cc(Cl)c(COC2CCCCC2)cc1O', 'Clc1cccc(Nc2cc(F)ccc2Oc2cccc(F)c2)c1', 'O=C(CNS(=O)(=O)c1ccc2ccccc2n1)N1CCN(S(=O)(=O)c2ccccc2)CC1', 'Clc1ccc(COc2nc(-c3cccc(Br)c3)co2)cc1', 'Clc1ccccc1OCc1csc([C@H]2CCC[NH2+]C2)n1', 'Clc1cc(Nc2n[nH]nc2C2CC2)ccc1Cl', 'Clc1cccc(/C=C/C=C/CC[NH2+]C2CC2)c1', 'O=[N+]([O-])c1cccc2ncccc12', 'O=[N+]([O-])c1cccc2ncccc12', 'Cc1c([N+](=O)[O-])cc(C(C)(C)C)nc1[N+](=O)[O-]', 'Clc1ccc(Br)cc1Sc1ncn[nH]1', 'O=[N+]([O-])c1c(OC[C@H]2CCCO2)[nH]c2ccc(Cl)cc12', 'Clc1nnn(-c2ncn[nH]2)c1-c1ccc(Cl)cc1', 'C=C1c2cc3ccccc3cc2C(=O)N1c1ccc(C)cc1', 'Oc1ccc(O[C@H](C2CC2)c2ccc(F)cc2F)cc1', 'Oc1cc(Cn2ncc(-c3cc([N+](=O)[O-])ccc3F)n2)ccc1Cl', 'O=[N+]([O-])c1c(Br)sc2ncn(Cc3cccnc3C(F)F)c12', 'O=C(c1ccco1)N1CCOc2cc(-c3ccccc3F)ccc21', 'O=C(CSCC(F)(F)F)N[C@@H](C(=O)c1cccc(Cl)c1)N1CCCC1', 'Oc1cc(-n2nncc2Br)c2ccccc2c1O', 'Clc1ccccc1Oc1cnc2nnnn2c1[O-]', 'Oc1cccc(Br)c1O\n\n', 'Oc1cccc(Br)c1O\n', 'Oc1cccc(Br)c1O', 'Clc1cc(Br)c2nc(Oc3ccccc3)cnc2n1', 'COCCn1ccc(C[NH2+][C@@H]2CCCc3c(C)cc(C)cc32)c1', 'Clc1nnnn1/C=C/C(F)(F)F', 'Clc1cccc([S@@]C2(COC(=O)Nc3ccc(-n4cccn4)cc3)CCCC2)c1', 'Clc1cccc([S@@](=O)c2ccc(Br)o2)c1', 'Clc1ccc(-c2cc(=O)oc(=O)c3ccccc23)cc1', 'Oc1cc(C=Nc2ccccc2)cc(I)c1O', 'CN=C([C@@H]1COC1)c1ccccc1C(F)(F)F', 'Clc1cc(Br)c2nc(-c3ccncc3)ncc2c1', 'Clc1cc(Br)c2nc(-c3cc[nH]c3)[nH]c2c1', 'Clc1cccc2c1C/C(=C\\c1ccccc1O)[C@@H]2c1ccccc1F', 'Clc1ccc(C(N)=NSc2ccc(F)cc2)c(Br)c1', 'Clc1ccc(-c2n[nH]cc2NC(=O)c2cc([N+](=O)[O-])ccc2Cl)cc1', 'Oc1ccccc1Oc1ccc(N2CCOCC2)cc1Cl', 'Clc1ccccc1Oc1cc(N2CCOCC2)ccc1O', 'Clc1cccc2nc(Nc3ccc4c(c3)OCO4)nc2c1', 'COc1nc2scnc2cc1CNCc1cc(F)c(C)cc1OC', 'NS(=O)(=O)C[C@@H]([NH3+])c1cccc(Cl)c1', 'Clc1ccc(Br)cc1NCC[NH+]1CCCC1', 'Clc1ccc(Br)c([C@@H]2CCC[NH+]2C[C@H]2CO2)c1', 'Clc1ccc(Br)c(NCC[NH+]2CCCC2)c1', 'O=C(CSCC(F)(F)F)NNC(=O)c1ccc(S(=O)(=O)N2CCCC2)cc1', 'Clc1ccc(Br)c(-n2ncc(C(F)(F)F)c2Cl)c1', 'O=[N+]([O-])c1cc(O[C@@H]2CCCC[C@@H]2CCC[NH+]2CCCC2)c(NC2CC2)cc1', 'Oc1cccc2cc(O[C@H]3CCC[NH2+]C3)ncc12', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccccc1', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccccc1', 'NS(=O)(=O)c1ccc(C(=O)N2CCN(c3cc(-c4cccc(F)c4)on3)CC2)cc1', 'Clc1cc(Br)c(C[NH2+][C@H]2CCc3nnsc32)cc1F', 'Clc1ccsc1Sc1nc(CSc2ccncc2)c[nH]1', 'Clc1ccc([C@@H](O)[C@@H]2[C@@H]3CC[C@H](C3)[C@@H]2O)cc1', 'Clc1ccc(Br)cc1-c1cc(Cl)cc2ccccc12', 'N=Cc1cccc2c1OCCN(C(=O)/C=C\\c1nc(C)co1)C2', 'Nc1cn[nH]c1NC(=O)[C@H]1Cc2ccccc2S1', 'N[C@@H](c1ncccc1F)[C@@H]1Cn2cnnc2C1', 'NS(=O)(=O)Cc1cc(Cl)ccc1-c1nc(-c2ccncc2)cs1', 'Clc1ccc(Br)cc1-c1cc(=O)oc2ccccc12', 'COCc1nc(C(=O)N2N=CN(CC#N)CC2)cs1', 'COc1nn(-c2c(NC(=O)c3ccccc3)c3ccccc23)cnn1C', 'NS(=O)(=O)N1CCC[C@H]1c1cc(F)cc(F)c1', 'Clc1ccccc1-c1nn2c(Cl)cnc2n1Cc1cc(Cl)ccc1Cl', 'OCSc1ccc2cc(Cl)ccc2c1', 'O=C(CCc1ccccc1)NCCc1nnc(-c2ccc(Cl)s2)n1-c1cccnc1', 'CNC(=O)CSc1nc(-c2ccccc2C)sc1C', 'Clc1ccccc1NC1C(=O)NC(=S)N1c1ccccc1', 'Clc1cccc2ncn(Cc3cc(F)cc(F)c3)c12', 'Clc1ccsc1C[N-]Sc1ccc(Br)cc1', 'Clc1ccc(-c2cc([N+](=O)[O-])ccc2OCC(F)F)cc1F', 'O=C(COc1cc([N+](=O)[O-])c([O-])cc1Br)NCc1ccc(Cl)cc1', 'Clc1cc(Br)c(S(=O)(=O)[C@@H]2CC[NH2+]C2)cc1F', 'OCc1nnc(-c2ccc(Cl)cc2F)[nH]1', 'O=[N+]([O-])c1c(Cl)ncc2ccc(Cl)cc12', 'Clc1nncc(-c2ccc(Cl)c(Cl)c2)n1', 'Clc1ccccc1Oc1cnc2cc(F)ccc2n1', 'Clc1ccccc1Oc1cc(Cl)c2[nH]nnc2c1', 'Clc1ccc(-c2nonc2)cc1F', 'Clc1ccc(-c2nonc2)cc1Cl', 'Clc1ccc(-c2nn(-c3ccccc3)[nH]c2=O)cc1F', 'Clc1ccc(-c2cc[nH]n2)c([N+](=O)[O-])c1', 'Clc1ccc(-c2cc(F)c(=O)[nH]c(=O)n2)cc1', 'Clc1cc(Cl)nc(-n2ccnc2)c1', 'Clc1cc(CCl)n2cncc2n1', 'Clc1cc(Br)cnc1-n1cccn1', 'COCC(=O)NC1(N(C)Cc2ccc(F)c(F)c2)CCC1', 'O=C([O-])/C(C=C(/C#N)C1CCCCCC1)NNc1ccccc1C#CCO', 'O=[N+]([O-])c1cc(Br)cc2cc(-c3ccc(Cl)c(Cl)c3)c(=O)[nH]c12', 'OCc1ncn(-c2ccc(Cl)cc2Cl)n1', 'Clc1ccc(-c2ccccc2Cl)c(-n2cnnn2)c1', 'N=Cc1n[nH]c(C(=O)N2CC[C@@H](C)[C@@H](O)C2)n1', 'Oc1cc(F)ncc1SCC(=O)Nc1ccc2c(c1)OCCO2', 'CCn1ncc2cccc(CC(=O)NCc3ccccn3)c2c1=O', 'ClCc1n[nH]cc1C[NH+]1CCN(c2ccc([N+](=O)[O-])cc2)CC1', 'Clc1ccc(-c2cc(Cl)n[nH]2)cc1CO', 'Clc1cccc2[nH]nc(N/C=C3/CS[C@@H](F)C[NH+]3)c12', 'Clc1ccccc1NS(=O)(=O)c1c[nH]c2cc(Cl)ccc12', 'O=[N+]([O-])c1cccc(OCCc2ccco2)c1', 'N#C[C@@]1([C@H](O)c2ccco2)CCC1(C)C', 'Clc1nnn(C(C)(C)C2CCSCC2)c1N1CCCC1', 'Oc1ccc(F)cc1N[C@@H]1CCCSC1=O', 'O=C(Cc1nn[nH]n1)c1ccc2c(c1)Cc1ccccc1C2', 'Oc1cc(F)nc(Sc2ccccc2Br)c1', 'CN[C@H](c1ccc(Cl)cc1Cl)c1cc(F)ccc1F', 'O=[N+]([O-])c1c(F)cc2[nH]ccc2c1Oc1ccccc1', 'O=C(/N=c1/occn1Cc1ccccc1)c1cccc(F)c1', 'OCc1nnsc1COc1cc(Cl)ccc1Cl', 'NS(=O)(=O)Cc1ccc([N+](=O)[O-])cc1', 'Clc1ccc(-c2cc([N+](=O)[O-])ncc2N2CCCC2)cc1Cl', 'Clc1cc(Br)cc(Oc2ccccc2Br)c1', 'Clc1cc(Br)cc(I)c1Oc1ccccc1', 'Oc1ccc(/C=C2/SC(=S)N(c3ccccc3)C2=O)cc1', 'Clc1ccsc1[C@@H]1Oc2c(nnc3c2SCC3)C1', 'O=[N+]([O-])c1cc(Cl)c2[nH]ncc2c1', 'Oc1ccccc1[C@@H]1CCCN1S(=O)(=O)/C=C/c1ccco1', 'Clc1cccc([C@@H]2[C@@H](C(=O)c3cc(Cl)cc(Cl)c3)NC2=O)c1', 'NS(=O)(=O)/C(Cc1ccccc1)c1cccc([N+](=O)[O-])c1', 'ClC(/N/c1c(Cl)cccc1Cl)(SCc1c[nH]c(=O)[nH]1)Cc1ccccc1', 'CN=C/C=C(\\C#N)C(=O)[C@@H](c1ccccc1)c1ccccc1', 'ClCc1ccc2nnc(-c3ccc(Br)c(F)c3)n2n1', 'O=C([O-])CS(=O)(=O)Cc1nccn1Cc1c(F)cccc1Cl', 'O=[N+]([O-])c1c(Br)cccc1N1CCCCCCCCC1', 'Clc1ccsc1Nc1cccc(Br)c1', 'Clc1nnnc(-c2cn(Cc3ccccc3)nn2)n1', 'O=[N+]([O-])c1ccncc1C(F)(F)F', 'Clc1ccsc1-c1n[n-]c(C(=O)N2CCCC[C@H]2c2ncccn2)n1', 'Clc1ccccc1NS[C@H](Nc1cccnc1Cl)c1ccccc1', 'Clc1ccc([C@@H](Br)c2ccc(Cl)c(F)c2F)cc1', 'Clc1cccc2c1Oc1ccccc1CC2', 'Oc1cc(-n2cnnn2)c2cnn(CC3CCCCC3)c2n1', 'NS(=O)(=O)/C(=O)c1cccc2c1[C@H]([C@@H]1CCCCN1C(=O)c1ccco1)N2', 'NS(=O)(=O)Cc1cccc(NC(=O)c2cnn(-c3ccccc3)c2)c1', 'Clc1ccc(Cc2cc[nH]c(=O)c2)cc1', 'Oc1cccc(CCc2cncc(-c3ccccc3)n2)c1', 'Clc1ccc(Br)cc1-c1cnncn1', 'OCc1n[nH]cc1-c1cccc(Cl)c1', 'Clc1cccc(-c2cncc(Br)n2)c1', 'ClCc1n[nH]cc1-c1cccc(Cl)c1', 'Clc1nnnn1/C(\\[O-])c1ccccc1C(F)(F)F', 'Oc1ccc(F)cc1COc1ccccc1', 'Oc1ccc(F)c(OCc2ccccc2)c1', 'O=C(/N=C\\c1c(F)cccc1Cl)c1cccc2cccnc12', 'Clc1cc(Br)c(N2CCN(Cc3cscn3)CC2)cc1F', 'Oc1cc(F)ncc1CS[C@@H]1C=C[C@H]([NH3+])C1', 'Oc1cc(-n2cccc2)ncc1S[C@@H]1CC[NH2+]C1', 'Nc1c(C2CC2)nc2scc(-c3ccccc3)c2c1C', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1-c1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1-c1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1-c1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1-c1ccccc1', 'Clc1ccc([C@@H](Br)c2ccc(Cl)cc2)cc1', 'Oc1cc(-n2nnnc2SSc2ccc(Cl)cc2[N+](=O)[O-])cc(Br)c1F', 'O=[N+]([O-])c1ccc2onc(Nc3ncccc3Cl)c2n1', 'O=C(c1cc(Br)cnc1Cl)N1CCC(c2ccccc2)=N1', 'O=C([O-])c1nc([C@@H]2CCCN(c3ccccc3)C2)nc2c(C)cccc12', 'O=C([O-])c1ccccc1C[C@@]1(O)C[NH2+]CCO1', 'Clc1ccccc1Oc1cc([N+](=O)[O-])ccn1', 'Clc1ccccc1Oc1cc(-c2ccncc2)no1', 'Clc1ccc(-c2cc(Cn3cnnn3)ccn2)cc1', 'Clc1cc(Cl)nnc1OCc1ccco1', 'Clc1ccccc1[C@H]1Oc2ccc(Cl)cc2CCN1c1ccc([N+](=O)[O-])cc1', 'ClCc1csc(COc2cccc(Br)n2)n1', 'O=C(CSCCOCc1ccccc1)Nc1n[nH]c2ccccc12', 'Clc1ccccc1Nc1nnc(-c2cc(Br)ccc2O)[nH]1', 'Clc1cc(Br)cc(F)c1C(=O)Nc1nccc(-c2cnccn2)n1', 'Clc1ccccc1-c1csc([C@H]2CCCC[C@@H]2C(=O)[O-])n1', 'Clc1ccsc1-c1nnon1', 'Clc1ccsc1-c1nnco1', 'Clc1ccccc1-c1nc(-c2cccs2)no1', 'Clc1ccc(-c2csc3s/cnc23)nn1', 'Clc1ccc(-c2csc3[nH]nnc23)s1', 'Clc1ccc(-c2csc3[nH]nnc23)s1', 'Clc1ccc(-c2csc3[nH]nnc23)cn1', 'Clc1ccc(-c2csc3[nH]ncc3c2=O)cn1', 'Clc1ccccc1CSc1nccc(C(F)(F)F)n1', 'Clc1ccccc1Oc1cc([N+](=O)[O-])c2ccccc2n1', 'O[C@@H](Sc1cccs1)C(=O)NNS(=O)(=O)c1ccccc1Cl', 'Clc1cc(/N=C\\Cc2ccccc2Br)cs1', 'Clc1ccsc1-c1cc2cccc(F)c2nc1NC1COC1', 'Oc1ccc(/C=N\\C(=O)COCc2ccc(F)cc2Cl)c(F)c1', 'O=C(c1cc(Cl)c2c(c1)OCCO2)N1CCN(c2ncccn2)CC1', 'O[C@@H]1CSc2ccccc2O1', 'ClC[C@@H]1COCCN1c1cccc(O)c1', 'O=[N+]([O-])c1c(OCc2ncon2)oc2ccc(Cl)cc12', 'Clc1ccccc1NS(=O)(=O)c1ccc(F)cc1', 'Clc1ccccc1NS(=O)(=O)c1ccc(F)cc1', 'Clc1ccccc1NS(=O)(=O)c1ccc(Cl)cc1', 'O=[N+]([O-])c1c(Cl)n2nnnc2n1-c1ccccc1', 'NS(=O)(=O)Nc1cc(C(=O)N2CCc3ccccc3C2)ccc1O', 'Clc1ccccc1N1N(C[C@@H]2CCCO2)C=C1c1cccc(Cl)c1', 'Clc1cc(Cl)nc(Cl)c1Cl', 'Clc1ccccc1[C@H](Cl)[C@H]1COc2ccccc21', 'Nc1nncn1[C@@H]1CC(=O)N(c2cccc(Cl)c2)C1', 'ClCc1cc(F)ccc1-n1cncn1', 'Oc1ccccc1[C@H]1O[C@@H](CO)[C@@H](O)[C@H](O)[C@@H]1O', 'OC[C@@]1(c2cccs2)NC(=S)N1Cc1cccs1', 'Cc1sn(-c2ccccc2)c1COC(=O)c1cnn(C)c1C', 'ClCc1csc(Cn2ccnc2-c2ccccc2F)c1', 'O=C(c1cccnc1SCC(=O)Nc1cccc(Cl)c1)N[C@@H]1CCCC[C@@H]1CO', 'OC[C@@]1(C(F)(F)F)c2ccccc2N=C(N2CCN(Cc3ccccc3)CC2)c1', 'Oc1cc(Br)cc(N2CCN(CC3CC3)CC2)c1', 'Clc1ccccc1O/Cn1c(=S)oc2ccc(Cl)cc21', 'Oc1cc(-n2cnnn2)cc(-c2ccco2)c1', 'O=C(CN(CCS)Cc1ccccn1)c1cccc([N+](=O)[O-])c1', 'Clc1ccc(-c2csc3scnc23)c[nH+]1', 'ClC(=S)Nc1ccc2c(c1)OCCO2', 'O=C([O-])[C@@H]1Cc2cc(-c3cccs3)[nH]c2N1', 'Clc1ccsc1N/N=C\\c1c[nH]c2ccccc12', 'Clc1ccc(-c2n[nH]c(NCc3cccc([N+](=O)[O-])c3)n2)cc1', 'O=C(c1cccs1)CNc1ccc(Br)cc1', 'Clc1cc(Cl)cn2ncnc12', 'Clc1ccccc1-c1cscc1-n1cncn1', 'N#C[C@@]12CCC(CCNC(=O)c3sc4ncc(Cl)cc4c3[C@H]1C(=O)c1ccccc1)C2', 'Oc1ccc([N+](=O)[O-])cc1NCc1cccnc1', 'Clc1cccc2c1nc(N1CC[C@@H](Oc3ccccc3)C1)OCCCO2', 'O=C(c1ccc[nH]c1=O)Oc1ccc(Br)c([N+](=O)[O-])c1', 'Clc1ccc(-c2csc3s/cnc23)c(Br)c1', 'O[C@@H](S[C@@H](Cc1ccccc1)c1ccccc1)c1ccco1', 'Nc1c(CSCc2ccccn2)cnn1C', 'O=[N+]([O-])c1cc(Br)c(NC2CC2)cn1', 'Oc1cc(F)c(Br)cn1', 'Clc1ccc(-c2cc[nH]n2)c(Br)c1', 'NS(=O)(=O)/N=c1\\sc(N)c(-c2ccc(Br)cc2)c(=O)c2ccccc12', 'Clc1ccccc1O/N=[N+](\\[O-])c1cccc(S(=O)(=O)N2CCCC[C@H]2C(F)(F)F)c1', 'Clc1ccc(Br)cc1S(=O)(=O)N1CCCC1', 'O=[N+]([O-])c1cc([C@H]2[C@@H]3OCC[C@H]3CC[C@H]2O)cc([N+](=O)[O-])c1O', 'Clc1cccc([C@@H]2O[C@@H](O)CO[C@@H]2c2cccc(OC(F)F)c2)c1', 'O=C(CSC(O)(CO)OCc1ccccc1)NCc1cccs1', 'Clc1ccc(Br)cc1-c1cc2ncccc2nc1Cl', 'Clc1ccc(-c2cccs2)nc1[N-]S(=O)(=O)c1ccc(Cl)cc1', 'Clc1cc(NC[C@@H]2CCS(=O)(=O)C2)ncn1', 'Clc1ccc(Br)cc1COC(=O)c1cccc(F)c1', 'NS(=O)(=O)/C=C/C=C/c1ccccc1', 'Clc1ccccc1NS(=O)(=O)c1ccc([N+](=O)[O-])cc1', 'Clc1ccc([C@@H](Cl)[C@H]2CN3CCC[C@@H]3CO2)cc1', 'Clc1ccsc1Sc1n[nH]c([C@H]2CCC[NH+]2Cc2ccco2)n1', 'Clc1ccccc1-c1noc(CSc2nnc(C3CC3)s2)n1', 'Oc1ccc(Oc2cc(-c3ccccc3)ncn2)cc1O', 'Oc1cc(F)ncc1COc1ccccc1F', 'C[C@@H](NCc1ccoc1Br)Cc1cccc(F)c1', 'Oc1cc(Cn2nc3ccccc3n2)ccc1Cl', 'Oc1cc(-n2ncc(Cl)c2Cl)ccn1', 'O=[N+]([O-])c1cc(Cl)nc2cnccc12', 'Clc1ccc(-c2nonc2)c(C(=O)[O-])c1', 'Clc1cc(Nc2nc[nH]n2)ccc1O', 'Clc1cc(CCl)nnc1-c1cnccn1', 'Clc1cc(Br)c2nc(OCc3ccccc3F)[nH]c2c1', 'Clc1nncn1c1c(Br)cc(I)cc1O', 'O=C(C1=c2ccccc2=[NH+][C@@H]1C(F)(F)F)N1CCN(Cc2ccc([N+](=O)[O-])cc2)CC1', 'Clc1ccc(Br)cc1[C@@H]1O[C@@H]2CCC[C@H]2[C@@H]1[NH3+]', 'Oc1ccc(/C=N/c2cccc(Br)c2)cc1', 'Clc1cc(Cl)nc2ncnc(Nc3ccc4c(c3)OCO4)c12', 'Oc1cc(F)ccc1NC[C@@H]1C[C@H]2CC[C@@H]1C2', 'O[C@@H]1Cc2ccccc2[C@@H]1Cc1c(F)cccc1F', 'Clc1cccc2cc(F)nnc12', 'Clc1cccc2nc(-c3cc(F)c(F)cc3Br)[nH]c12', 'Clc1cc(Br)cnc1Oc1cccc(Br)c1', 'Clc1ccc(-c2cc(Cl)cs2)cc1F', 'Clc1ccc(Br)cc1OC[C@H]1CCCO1', 'Clc1cc(F)ccc1C[NH2+]C1C[C@@H]2CC[C@@H](C1)C2', 'Clc1cc(Cl)c(C[NH2+]CC2CCCCC2)cc1F', 'O=C(CNc1cc2c(cc1Cl)OCO2)c1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnc2cc[nH]c12', 'Clc1ccc(-c2cccnc2)c([C@H]2N=c3ccccc3=[NH+]2)c1', 'N=C1/C(=C/c2ccc(OCc3cc(C(=O)NCc4cc[nH+]cc4)ccc3)cc2)C(=O)SC1', 'O=C(CSCCO)Nc1ccccc1NCc1cccc(Cl)c1', 'O=[N+]([O-])c1ccncc1Nc1ccc([N+](=O)[O-])cc1', 'Clc1cccc(NC2=[NH+]C[C@@H]3CCCC[C@@H]3C2)c1', 'O=C(CSc1nc2ccccc2c(=O)n1Cc1ccccc1)NNC(=O)c1ccccc1F', 'Clc1ccc(Br)c([C@@H]2C[C@@H]2c2ccccc2)c1', 'COc1ncncc1CNC(=O)C(=O)Nc1cccc(OC)c1', 'Cc1cc(=O)oc2cc(NC(=O)/C=C/c3ccccc3)ccc12', 'CC[S@](=O)[C@@H]1CCCC[C@@H]1NC(=O)c1cccnc1', 'Oc1ccccc1/C=C/C[NH+](Cc1ccc(Br)cc1C(F)(F)F)Sc1ncccn1', 'Oc1cc(OC(C(=O)[O-])c2ccc(C(C)CO)cc2)cs1', 'O=C(CNS(=O)(=O)c1ccccc1C(F)(F)F)N[C@H](c1ccccc1)c1ccc([N+](=O)[O-])cc1', 'O=C(c1cn2ccsc2c1)N1CCC[C@H]1c1cccc(F)c1', 'O=C(c1ccsc1C#N)N1CCN([C@@H]2C=CC[C@@H]2C(=O)c2ccccc2)CC1', 'Oc1ccc(Oc2ccc(C(=O)[O-])cc2)cc1', 'Oc1ccc(F)cc1N(CCCC#N)C1CC1', 'Oc1cc(-n2nnnc2SCC(=O)c2ccccc2)coc1=O', 'Clc1ccc(Br)cc1S(=O)(=O)N1CCOCC1', 'O=C(c1cc([N+](=O)[O-])ccc1F)N1CCC([NH2+]CCN2CCCCC2)CC1', 'COc1nn(C)cc1C(=O)N1CC[C@H](C#N)C[C@@H]1c1ccc(C)o1', 'O=C(c1c[nH]c(=S)s1)N1CCC(OCc2cccc(F)c2)CC1', 'Oc1ccc(F)cc1[C@H](CO)NC1CC[NH+](C2CC2)CC1', 'Oc1ccc(F)c(CNCCN2C(=O)CCCC2=O)c1', 'Oc1cc(F)cc(CNCC2(N3CCOCC3)CC2)c1', 'ClC[C@H]([NH2+]Cc1cc(O)ccc1O)N1CCOCC1', 'O=C(c1cccc(I)c1)[C@H]1COc2ccccc2C1', 'ClC(c1cc(Cl)on1)c1cc(I)cc(O)c1', 'O=[N+]([O-])c1c(Br)c(O)nc2ncccc12', 'O=C(CNS(=O)(=O)c1ccc([N+](=O)[O-])cc1)NCc1ccccc1', 'Oc1cc(F)ccc1[C@@]1(COc2ccccc2F)CCO1', 'Clc1ccc(-c2nn(-c3ccc[nH]c3=O)cc2F)cc1Cl', 'O=C(CNS(=O)(=O)c1cccs1)NC1CC1', 'Clc1cc(Cl)nn1Cc1cnc2ccccc2c1[O-]', 'COc1ncc2nn(C)c(=O)c(C=O)c2c1C', 'O=C(c1ccco1)c1cc(S(=O)(=O)NC[C@H]2CCCO2)c(F)cc1F', 'O=C(CS(=O)(=O)c1cc(F)ccc1F)N[C@@H](CO)c1ccco1', 'C=C(Br)Cn1c(SCC(=O)N[C@H](C)c2ccccc2)nn(-c2ccccc2)c1=S', 'NS(=O)(=O)/C(C#N)=C\\c1ccc(-c2ccc(Cl)cc2)cc1', 'OCSC1=N[C@@H](COc2cccc(Cl)c2)N(C(=O)c2ccc([N+](=O)[O-])cc2)C1', 'Cc1nonc1C(=O)Nc1cc(C(=O)NCCC#N)ccc1Br', 'Clc1cccc(Nc2cccc(COc3ccccc3)c2)c1', 'Oc1cccc2cc([C@@H]3CCC[NH+]3C3CC3)ccc(=O)c12', 'O=C(COc1cc(Br)ccc1Br)Nc1ccc(Cl)cc1', 'Clc1nnnn1/C(=C\\c1nc[nH]n1)c1ccc(I)cc1', 'ClC[C@@H]1CNC(=O)c2ccccc21', 'O=C(/N=C/N=C/c1cc2ccccc2o1)c1ccc(Br)cc1', 'Clc1nnnn1/C(/[O-])CC(=O)Nc1cccc(C(F)(F)F)c1', 'Clc1ccccc1-c1cscc1Cn1cncn1', 'Clc1ccccc1-c1csc(Cn2ccnc2)n1', 'Clc1ccccc1N1N=C2SCC[C@@H]2C1=O', 'Oc1ccc([N+](=O)c2onnc2Cn2cncn2)cc1', 'Oc1ccc(F)c(Oc2cc(F)ccc2F)n1', 'Oc1cc(OC[C@@H]2CCCO2)ccc1NCc1ccc(F)cc1', 'O=[N+]([O-])c1ccnc2ccc(F)cc12', 'O=[N+]([O-])c1ccnc2ccc(Br)cc12', 'O=[N+]([O-])c1c(Br)cnc2ccc(F)cc12', 'Clc1ccc(-c2ccnc3ncccc23)cc1F', 'Clc1cc(F)c(Cl)cc1Oc1ccccc1F', 'Clc1cc(Br)c2nc(-c3ccccc3Cl)oc2c1', 'Clc1cc(Br)c2nc(-c3ccc(Br)cc3)oc2c1', 'Clc1cc(Br)c2nc(-c3cc(F)ccc3Cl)oc2c1', 'NS(=O)(=O)Cc1nc(Cc2ccccc2)cs1', 'NS(=O)(=O)CCc1nc(-c2ccccc2)cs1', 'Clc1ccc(-c2nonc2[S-])nn1', 'O=C(/C=C/c1cnccn1)Nc1cccc(Oc2ccccc2)c1', 'Clc1cc(Br)c2nc(-c3cccs3)[nH]c2n1', 'O=C(CNc1c(Cl)cccc1[O-])Nc1ccc(C#CCCO)cc1', 'Nc1n[nH]c(-c2ccc(F)cc2)c1C(=O)Nc1ccc2c(c1)COC2', 'O=[N+]([O-])c1c(F)cc2nc(N3CCC(O)CC3)n[nH]c12', 'O=C(CC[C@@H]1CCCO1)NCCc1cccc([N+]2=CCCC2)c1', 'Clc1cc(Cl)c2sccc2c1Oc1ccccc1', 'Clc1ccc([C@@H](Cl)[C@@H]2CCCOCC2)cc1', 'Oc1cccc(CCl)c1F', 'ClCc1cccc(I)c1O', 'Clc1ccc(-c2csc3s/cnc23)nc1', 'Clc1ccc(-c2csc3s/cnc23)nc1', 'Clc1ccccc1[C@H]([NH2+]CCO[C@H]1CCS(=O)(=O)C1)c1nccs1', 'Clc1ccccc1C1=CSCC[NH2+]CC1', 'Oc1cc(F)cc2c1NC(=O)N2Cc1cc(Cl)sc1Cl', 'O=C(/C(c1cccs1)=C/c1ccc(O)cc1)c1ccc(S(=O)(=O)N2CCCC2)cc1', 'O=C([O-])/C=C/c1ccc(-c2ccccc2[N+](=O)[O-])cc1O', 'Clc1cc(F)cc(NNC(=O)COc2cccc(Br)c2)c1', 'Oc1cccc(Oc2ccccc2NCc2ccc(C(F)(F)F)cc2)c1', 'NS(=O)(=O)/C=C\\C(=O)c1ccc([N+](=O)[O-])cc1C(F)(F)F', 'Oc1cc(Br)cnc1OCc1ccnnc1Cl', 'Oc1ccc([N+](=O)[O-])c(Sc2cccc(Br)c2)c1', 'Clc1ccccc1OC1(Br)CC1', 'ClC[C@H]1OC(c2ccccc2)=C1Cl', 'Clc1ccsc1CNc1ccc(F)c(C(F)(F)F)c1', 'Nc1c(/C=N/c2ccc(Cl)cc2)cc(Br)nc1CO', 'Clc1ccc(COc2nc(F)cc(Br)c2)c(F)c1', 'Clc1ccc(COc2nc(Cl)cc(Cl)c2Cl)cc1', 'N#C[C@H]1C(=O)NC[C@@H](C(=O)c2ccccc2)N1CNc1cccc(Cl)c1', 'Oc1cccc(-c2csc(-c3ccccc3)n2)c1', 'Clc1cccc2ccc(Cl)cc12', 'Clc1cccc2ccc(Cl)cc12', 'Clc1cccc2ccc(Br)cc12', 'Clc1cccc2cc(F)ccc12', 'Clc1cccc2cc(F)ccc12', 'Clc1cccc2cc(Br)ccc12', 'Clc1cccc2cc(Br)ccc12', 'Clc1cccc2cc(Br)ccc12', 'Nc1[nH]ncc1-c1ccc([N+](=O)[O-])cc1', 'O=[N+]([O-])c1cc(Br)ccc1Sc1ccc(F)cc1O', 'Clc1ccc(Br)cc1-n1cncc1Cn1ccnn1', 'Clc1ccc(-c2csc3sccc23)cc1F', 'Clc1ccsc1/CCc1cnc(-c2cccs2)n1C1(C#N)CCCC1', 'Clc1ccc(Cc2nnc3scnc3n2)cc1', 'Clc1ccccc1-c1cc(C2CCOCC2)ncn1', 'Oc1cc(COC[C@H]2CCCCOC2)cc([N+](=O)[O-])c1Cl', 'O=[N+]([O-])c1cc(OCOCC[C@H]2CCCCO2)ccc1Cl', 'Oc1cc(O)ccc1CN1CCN(S(=O)(=O)c2cccc(Cl)c2)CC1', 'Clc1cc(/C=N/Oc2ccccc2Cl)ccc1OS(=O)(=O)c1ccccc1', 'COc1ncc(SCC#N)c(-c2ccc(Cl)cc2Cl)n1', 'O=C(Cc1noc(C2CCCCC2)n1)N[C@@H]1CCSc2ccccc21', 'O=[N+]([O-])c1cc(Br)c(Cl)cc1[C@@H](O)c1ccccc1', 'Clc1ccc(C[NH+](CCCn2cccc2)C2CC2)s1', 'Clc1ccc([C@@H](Nc2ccc(Br)cc2)C(F)(F)F)cc1Cl', 'Clc1nnnn1-c1cn[nH]c1', 'Clc1ccccc1Nc1cc(C2CC2)ncn1', 'O=C(/C(CO)=O)c1cc(F)ccc1Oc1ccccc1', 'O=C(c1cn2ccsc2n1)N1CC(c2ccccc2)=N1', 'O=[N+]([O-])c1ccc(COc2ccc(CC[NH+]3CCOCC3)cc2)cc1', 'Clc1ccc(-c2csc3[nH]c(N[C@@H]4CCSCC4)cc23)cc1', 'O=C(/N=c1/occ2ccccc2c1Cl)c1ccc2c(c1)OCCO2', 'ClC[C@H](Br)[C@@H]1CCCN(S(=O)(=O)c2cccs2)C1', 'Oc1cc(F)nc(SCC(F)F)n1', 'Clc1ccsc1[C@@H](O)c1ccncc1', 'Clc1ccc(Cc2nccs2)nc1', 'Clc1ccc(Cc2csnn2)cc1', 'O=C(/C(c1cccs1)C(=O)Nc1ccc(F)c(F)c1)c1cccs1', 'Clc1ccccc1-c1csc(/C=C\\c2cccc(O)c2)n1', 'Cc1c([N+](=O)[O-])cccc1NC(=O)c1ccc(Cl)cc1', 'Clc1cccc2cc(C[NH+]3CCCC3)ccc12', 'Clc1cccc(Br)c1O/C=C/C(=O)c1ccc(Cl)cc1Cl', 'Clc1ccsc1[C@H]1[NH+]=c2ccccc2=[NH+]1', 'O=C(/N=C/C(=O)Nc1cc(C(F)(F)F)ccc1Cl)c1c[nH]c2ccccc12', 'Oc1cc(-c2noc(C3CC3)n2)ccc1SCC(=O)Nc1cccc(Cl)c1', 'Oc1ccc(F)cc1C[NH2+]Cc1cc(Cl)ccc1O', 'O=[N+]([O-])c1ccc2c(c1)CCN(Cc1ccc([O-])cn1)CC2', 'Clc1cc(NSc2nnc(-c3ccccc3)n2Cc2ccco2)n[nH]1', 'Clc1ccc(Br)cc1OS(=O)(=O)c1cccc(Br)c1', 'Clc1ccc(Br)cc1OS(=O)(=O)c1c(Cl)cccc1Cl', 'Clc1ccccc1-c1csc(Cc2cccnc2)n1', 'Cc1sncc1SCC(=O)N1CCN(c2cccc[nH+]2)CC1', 'O=[N+]([O-])c1ccc(F)cc1NC(=O)COC(=O)c1ccc2c(c1)OCO2', 'Clc1cccc2cc(Br)sc12', 'Clc1cccc2cc(Br)sc12', 'ClC(c1cc2ccccc2oc1=O)c1cccnc1Sc1ccc(F)cc1', 'O=C(c1cc(F)ccc1F)N(Cc1ccccc1)Cc1cccs1', 'Clc1ccc(Br)c(Sc2nccnc2N2CCNC2=O)c1', 'O=C(/C=C\\c1n[nH]c(=O)[nH]1)c1ccc2ccccc2c1', 'Clc1cc(/C=N\\O[C@H]2c3ccccc3O2)ccc1[O-]', 'NS(=O)(=O)Nc1cc(S(=O)(=O)C=O)ccc1C(F)(F)F', 'ClCc1ncoc1Cn1c(=O)oc2ccccc21', 'Clc1cccc(F)c1-c1cc(Cl)cc(Cl)c1', 'O=C(c1cc(F)c2[nH]ccc2n1)N1C[C@@H]2CC[C@@H](C1)N2C(=O)Nc1cccnc1Cl', 'O=C(c1cncc(F)c1)N1CCc2sccc2[C@@H]1c1cccc(Br)c1', 'O=[N+]([O-])c1cccc(NC2CC2)c1', 'Clc1cc(/C=N/O[C@@H]2CCCCNC2)cs1', 'Clc1ccsc1-c1c[nH]c(=O)[nH]c1=O', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccccc1[N+](=O)[O-]', 'Clc1ccc(Br)cc1-c1ccsc1', 'Clc1ccc(Br)cc1-c1ccsc1', 'O=C(Cc1cc2n(n1)COCCO2)[C@H]1Cc2ccc(F)cc2O1', 'O=[N+]([O-])c1ccc2oc(SCc3ccc(Br)o3)nc2c1', 'Oc1ccc(F)c(/N=C/c2ccn(-c3ccccc3)n2)c1', 'Clc1cc(Cl)ncc1-c1cccc2c1OCCCO2', 'O=[N+]([O-])c1c(-c2ccoc2)cccc1[N+](=O)[O-]', 'COc1nnn(-c2ccc(NC(=O)[C@H](C#N)c3cccnc3)cc2)c1', 'Clc1nnnn1/C(Cl)Cc1cccc(Br)c1', 'Clc1cccc(/C(Cl)/N=C/C=C\\c2ccco2)c1', 'Oc1cc(C(=O)N[C@@H]2COc3ccc(Cl)cc32)cs1', 'ClC[C@@H]1C(=O)N2CCCNC2=C1C(=O)c1cccc(F)c1', 'Clc1ccccc1Nc1cncc(C(F)(F)F)n1', 'O[C@@H]1[C@@H](C[C@@H]2CCCO2)C(=O)C[C@@H]1c1ccc2c(c1)OCO2', 'Clc1nnnn1c1nc(O)c(Cl)c([O-])c1Cl', 'Oc1cc(-n2nnnc2N2CCN(c3ccc(Cl)c(Cl)c3)CC2)ncn1', 'Clc1nncc(F)c1C(F)(F)F', 'Clc1cccc2[nH]nc(S/C=C/C[NH3+])c12', 'Clc1ccc(CO)c(-c2cc(Cl)cn3ncnc23)c1', 'O=[N+]([O-])c1c(Cl)cccc1C(=O)Nc1ncc[nH]1', 'ClCc1nonc1CN1C(=O)c2cc(Cl)ccc2S1(=O)=O', 'Clc1ccsc1N/C=C1\\NC=CCC1', 'Cc1ncsc1CN(C)c1ccc(C(=O)N2CCCC2)cc1', 'O=C(CS(=O)(=O)[N-]c1cc(F)ccc1SCc1cccs1)Nc1ccccc1', 'Oc1cc(O[C@@H](c2ccccc2)C(F)(F)F)ccc1Cl', 'NS(=O)(=O)C[C@H]1Nc2cc(Cl)ccc2N2CCCCC[C@@H]12', 'Clc1cc(Br)ccc1[C@H]1C[NH+]2CCCC[C@H]12', 'Clc1cc(Br)c(SCC(F)(F)F)cc1O', 'NS(=O)(=O)Cc1ccc(Cl)cc1', 'COc1ncc2nc(N/N=C/c3ccc(C#N)cc3)c(CC)nc2c1C', 'OCSCc1nc2cc(Cl)ccc2nc1Cl', 'N#Cc1coc(S(=O)(=O)Nc2c(-c3ccccc3)cnn2-c2ccc(F)cc2)c1', 'O[C@@H]1[C@@H](CO)CN(c2ccc(Cl)cc2)C1', 'Clc1ccc(-c2ccccc2Cl)cc1NCC(=O)[O-]', 'Oc1cc(O)c2[nH]cnc2c1/N=N/c1ccccc1', 'O[C@@H](Cn1cccn1)[C@H]1CSCCS1', 'Clc1cccc(CC/C([O-])=C2/NC=C/c3ccccc32)c1', 'O=C(c1c[nH]cn1)O[C@@H]1CCN(c2cccc(F)c2)C1', 'Clc1nnnc(NNc2ccc(C(F)(F)F)cc2)n1', 'Oc1ccc(/N=C/c2cccc(C(F)(F)F)c2)nc1', 'Clc1nnc(Sc2cccc(I)c2)nc1N1CCCC1', 'O=[N+]([O-])c1ccncc1[C@@H]1SCCCCc2ccccc21', 'Clc1cccc(/N=N/c2cccc(Oc3ccccc3)c2)c1', 'Clc1ccccc1N1N=C(C(=O)Nc2ccc([N+](=O)[O-])cc2)SC1=S', 'Clc1ccc(C2=CSCCN2)cc1[N+](=O)[O-]', 'Clc1ccsc1S(=O)(=O)N1C[C@@H](CO)C1', 'Clc1ccc(Br)cc1-c1cnc(C2CC2)nn1', 'Clc1cc(C[C@@H]2Cc3ccc(Cl)cc3O2)n[nH]1', 'Clc1cccc2nn(S[C@@H](Cc3cccnc3)c3ccccc3)nc12', 'Clc1ccc(COc2ncccc2F)cc1Br', 'Clc1cc(F)cc2nnc(N[C@@H]3CCCOC3)n12', 'O=C(c1ccccc1Sc1ccccn1)N1CCN(c2nccs2)CC1', 'Clc1ccc(Br)c(S(=O)(=O)N2CCSCC2)c1', 'COc1nc2sn(-c3ccccc3)c(=O)c2c2c1CCCCCC2', 'NS(=O)(=O)Nc1cc(Cl)ccc1C(=O)N(CCn1cccn1)C1CC1', 'Clc1ccccc1N1Nc2c(ccc(Br)c2F)C1', 'N[C@@H]1C[C@@H]([NH3+])CN1Cc1cccc(F)c1', 'O=C(CS(=O)(=O)Cc1cccc(Cl)c1)Nc1c(F)cccc1F', 'Clc1ccsc1NS(=O)(=O)c1ccc(C(=O)[O-])cc1', 'Oc1cccc(-n2c(S[C@H]3CCOC3)nc3ccccc32)c1', 'Oc1ccccc1-c1n[nH]c(Nc2cc(F)c(Cl)cc2Cl)n1', 'Clc1ccccc1Oc1cc(-c2n[nH]c(=O)[nH]2)ccc1O', 'Clc1cc(/N=C/c2nc3ccccc3[nH]2)c([N+](=O)[O-])cc1O', 'Clc1ccc(-c2n[nH]c3ccc(OC(F)(Cl)F)cc23)cc1', 'Oc1cc(F)ncc1S(=O)(=O)c1ccc(F)cc1', 'Oc1cc(F)nc(S/C=C(c2ccccc2)c2ccc(Cl)cc2)c1', 'O=C(c1cn2ccsc2n1)N1CCC(Oc2cc(S(C)(=O)=O)nc3ccccc23)CC1', 'O=[N+]([O-])c1ccc(Cc2nc(-c3ccco3)no2)c(F)c1', 'Clc1cc(Br)cc2nnnn12', 'Clc1ccc(-c2csc3[nH]c(N4CCCC4)nc23)cc1Br', 'Clc1ccc(-c2csc3[nH]ncc32)cc1F', 'Clc1ccc(-c2csc3[nH]ncc23)cc1F', 'Clc1ccc(-c2csc3[nH]ncc23)c(F)c1', 'Clc1ccc(-c2csc3[nH]cnc32)cc1Cl', 'Clc1ccc(-c2csc3[nH]cnc23)cc1Cl', 'O=C(C1CC=[NH+]C1)N1CCc2sccc2-c2cc(Cl)ccc21', 'Clc1cc(N2CCN(c3ccc(F)cc3)CC2)ncn1', 'Clc1ccccc1NC(=S)Nc1cccc(Cl)c1', 'ClCc1cc(O[C@@H]2CCc3ccccc32)c(F)cc1F', 'Clc1cc(F)ccc1/C=N/N[C@@H]1CCC[C@]12CCC[C@H]2[NH3+]', 'CNC(=O)c1ccc(NC(=O)c2sc(C(=O)N3CCCCC3)cc2C)cc1', 'Nc1c(Cl)nsc1NCc1csc2ccccc12', 'Oc1cccc(Oc2cccc(COc3cnccn3)n2)c1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1Cl', 'Clc1ccc(-c2csc3[nH]nnc23)cc1Cl', 'Clc1cccc(-c2cccc(I)c2)c1', 'Clc1ccccc1NC(=O)/C=C/c1ccc([N+](=O)[O-])cc1', 'Oc1ccc(F)cc1OC[C@H](O)c1ccc(Cl)cc1', 'Clc1nncn1CCNC(=O)Nc1nnc(-c2cccc3cccnc32)s1', 'Oc1ccc(Oc2ccc(NC(=O)Nc3cc(Cl)ccc3O)cc2)cc1F', 'Clc1nnnn1/C([O-])=C/c1ccc2ccccc2c1', 'NS(=O)(=O)Nc1cccc(NC(=O)c2cccc(F)c2)c1', 'ClCc1nnc(Sc2ccc(Cl)cc2)[nH]1', 'Clc1ccc(-c2cc(Br)co2)cc1F', 'Clc1ccc(-c2nonc2Cc2cccc(Br)c2)cc1', 'Clc1ccc(-c2cc(S(=O)(=O)C[C@@H](O)c3ccncc3)no2)cc1', 'ClCc1nc(Oc2ccc(Cl)cc2)nnc1-c1cccc(Cl)c1', 'Oc1ccccc1[C@H]1CC[NH2+][C@@H]2[C@H]1c1ccc(F)cc12', 'O=C(c1cn2ccsc2n1)[C@@H]1C[C@H]1c1ccccc1F', 'O=C(CSCc1ccc(Cl)c(Cl)c1)NNC(=O)c1ccc(S(=O)(=O)N2CCCC2)cc1', 'NS(=O)(=O)Cc1ccc(NC(=O)[C@H](O)c2cccc(Cl)c2)cc1', 'ClCc1nnsc1-c1cnn(-c2ccccc2)c1', 'Clc1cc(COCCO)cs1', 'Clc1ccc(Br)cc1NCCN1CCOCC1', 'Clc1ccccc1CNc1nc2ccc(Cl)cc2[nH]1', 'Oc1cccc(-n2ccc3ncnn32)c1', 'O[C@@H]1c2ccc(Br)cc2C[C@H]1CC(=O)Nc1ccc(OCC=O)cc1', 'Clc1cc(/C=C(\\C#N)C(=O)Nc2ccc(I)cc2)ccc1Cl', 'O=C(Cc1conn1)Nc1ccc2ccccc2c1', 'OCc1nc(Cl)cc(Oc2ccc(Cl)cc2Cl)c1', 'CCc1nc2n(n1)C=CC(C(=O)Nc1nc(C(C)(C)C)cs1)C(=O)C(C)(C)C2', 'Oc1cc(C=O)on1', 'Oc1cc(Br)c(Cn2ccnn2)c(Cl)c1', 'O=C(c1c[nH]cn1)c1cccc(CN2CCC[C@H](O)C2)c1', 'COc1ncncc1CCCNC(=O)c1cc(-c2ccco2)on1', 'Oc1cc(-n2cncn2)ccc1NN1CCOCC1', 'Clc1cccc2c1CNC(=O)C[C@@H]2c1ccccc1C(F)(F)F', 'Oc1cc(F)ncc1CNc1cc(C(F)(F)F)c2c(c1)CCCC2', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccccc1O', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccccc1Cl', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccc(F)cc1F', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccc(F)cc1F', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccc(Cl)cc1', 'Clc1ccc(Br)cc1OS(=O)(=O)c1cc(F)ccc1F', 'Clc1ccc(CNS(=O)(=O)c2c(Cl)cccc2Cl)cc1', 'Clc1ccccc1CSc1nnc(SCc2cccc3ccccc23)n1C1CCCC1', 'O=C(COCc1cc(F)ccc1F)Oc1ccccc1', 'OC1=C[C@@H](C(=O)Nc2ccccc2O)[C@@H]1C', 'O=[N+]([O-])c1cccc2c1[C@H](CO)C2', 'OCS(=O)(=O)c1ccccc1F', 'O=C([O-])[C@H]1OC(c2ccc([N+](=O)[O-])cc2)C(=O)N=C1N1CCCCC1', 'Clc1ccc(Br)c(Cl)c1N1CCC[C@H]([NH3+])C1', 'Clc1nncc(S/c2ccc(F)cc2F)n1', 'O=C([C@H](Cl)Br)c1ccccc1', 'OCc1nccs1', 'Clc1ccc(Br)c(F)c1Oc1cccc(CO)c1', 'Oc1ccc(O[C@H](CO)C(F)(F)F)cc1', 'Clc1cccc(Sc2nnc(NC[C@@H]3CCCO3)s2)c1', 'Clc1ccsc1S(=O)(=O)[N-]c1cccc(C(=O)NOc2ccccc2)c1', 'O=C(CSCCN1C(=O)COc2ccccc21)Nc1nccs1', 'Clc1nncc(N2CCSC[C@H]2Br)n1', 'Clc1nnnc(C(=O)Oc2ccccc2F)c1N', 'Clc1nnn(-c2ncnc(-c3ccncc3)n2)c1Cl', 'Clc1nnn(-c2ncn[nH]c2=O)n1', 'Clc1nnn(-c2ncc3ccccc3n2)n1', 'N#Cc1n[nH]nc1C[NH+]1CCC[C@@H]1c1cccc(Cl)c1', 'Clc1nnc(N2CC=C(n3cc(-c4ccccc4)s3)CC2)o1', 'OC[C@@H](c1ccco1)N1CC[C@@H](CS(=O)(=O)c2cccc(F)c2)C1', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1Oc1ccc(F)cc1', 'Oc1cc(O)c(C(=O)Nc2ccc3[nH]nnc3c2)cc1Cl', 'O=[N+]([O-])c1c(Cl)nnc2[nH]nc(-c3ccc(F)c(F)c3)c12', 'O=[N+]([O-])c1c(Cl)nsc1Oc1ccc(F)cc1F', 'OCc1nnsc1[C@H]1CCCN1C(=O)c1ccccc1', 'Oc1ccc(F)cc1-c1csc([C@@H]2Oc3ccccc3O2)[nH+]1', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(O)c(=S)n1Cc1cccs1', 'O=[N+]([O-])c1ccnc2c1CCC[C@@H]2Nc1ccc(Cl)c(F)c1', 'O=[N+]([O-])c1cc(N[C@H]2CCc3c(Br)cccc32)ncn1', 'O=C(c1c[nH]c(-c2ccsc2)n1)N1CCC[NH2+]CC1', 'O=C([C@H](Cl)c1nc2ccccc2s1)[NH+]1CCCCC1', 'O=[N+]([O-])c1ccnc2c1C[NH+](Cc1ccc(C(F)(F)F)c(Cl)c1)CC2', 'Oc1ccc([N+](=O)C=NN2C(=O)N[C@](c3ccccc3)=N2)cc1', 'Oc1cccc([C@H](OCC[NH3+])C(=O)[O-])c1', 'Cc1c([C@H](C)NC(=O)c2ccc3c(c2)OCCO3)cccc1[N+](=O)[O-]', 'Clc1cccc2ncn(C(Cc3ccco3)c3ccccc3)c12', 'ClC[C@@H]1C[C@H]1c1cc(I)c(Cl)cc1Cl', 'N#C[C@H](F)c1ccccc1OCc1ccc(Br)cc1', 'Oc1cc(-n2nc(Cl)cc2C(=O)c2cc(Cl)ccc2Cl)ccc1O', 'O=[N+]([O-])c1cc(F)ccc1N1CCC[C@H]1c1ccco1', 'N[C@@H]1Oc2ccc(C(=O)Nc3ccccn3)cc2C(=O)N1c1ccccc1Cl', 'O=[N+]([O-])c1c(Cl)nc(Cl)cc1Oc1cccc(Cl)c1Cc1ccccc1', 'O=C([O-])[C@H]([C@@H](C(=O)n1cncn1)c1ccccc1)c1cccc(Cl)c1', 'Clc1cc(N2CCC[C@@H]2c2nccs2)ccc1F', 'Clc1cccc(C(Br)CCl)c1', 'Clc1ccsc1SC[C@@H](O)c1cccc(Br)c1', 'Oc1cc(F)c(S(=O)(=O)[N-]c2ccc3c(c2)OCCO3)cc1C(=O)[O-]', 'Clc1cc(N2CCc3ccccc3C2)ccc1O', 'Cc1ncsc1C(=O)N1CC=C(c2ccc(Cl)cc2Cl)CC1', 'O=C(c1ccc(F)c(F)c1)N1CCSC[C@@H]1C(=O)[O-]', 'O=[N+]([O-])c1cccc2c1N[C@@H](C(F)(F)F)[C@H]1CC=C[C@@H]21', 'OCc1nc(-c2ccccc2Br)cs1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc([N+](=O)[O-])c1-c1cccs1', 'Clc1cccc(C2=NO[C@H](c3cccnc3)C2)c1', 'Oc1cc(F)ccc1C[NH2+]Cc1ccc(F)cc1F', 'Clc1cc(/C=C2/S[C@H]3CS(=O)(=O)C[C@@H]3N2c2ccccc2)ccc1[N+](=O)[O-]', 'O[C@@H](c1cc(F)cc(F)c1)c1nc(C2CC2)ns1', 'O=C(CS(=O)(=O)[N-]C1CCCC1)Nc1nc2ccccc2s1', 'ClCc1cccc(OCCc2ccccc2)c1', 'C[C@@H](NS(=O)(=O)c1ccc(F)c(F)c1)c1nc(-c2ccccc2)no1', 'Clc1ccccc1NC(=S)Nc1ccc(F)cc1', 'Clc1nncn1c1c(F)cc(F)cc1F', 'Clc1cc(Cl)nnc1Cl', 'Clc1cc(Cl)nnc1Cl', 'Clc1cc(/N=C\\c2ccc(F)cc2F)c2[nH]cnc2n1', 'Clc1cc(Cl)nc(CSc2cc(F)ccc2Cl)c1', 'N#Cc1c(F)cccc1NC(=O)N[C@H](CO)Cc1ccccc1', 'Clc1cccc(CC2(CO)OCCO2)c1', 'N=CC1CCN(C(=O)Nc2c(Cl)cccc2Cl)C1', 'Clc1ccccc1NS(=O)(=O)c1c[nH]c2nccc(Cl)c12', 'Clc1ccc(-c2cc([N+](=O)[O-])c([O-])nn2)cc1', 'O=[N+]([O-])c1c(F)cc2[nH]ccc2c1Cl', 'O=C(c1cnn2cc([N+](=O)[O-])cnc12)N(Cc1ccccn1)S(=O)(=O)c1ccc(F)cc1', 'Clc1ccc(-c2cccs2)[nH]c1=O', 'Clc1ccc(-c2cccs2)[nH]c1=O', 'Clc1ccc(-c2cccs2)[nH]c1=O', 'O=[N+]([O-])c1c(F)ccc2c1N=C(c1ccccc1)OC2=O', 'CN=N[C@H]1CC(c2ccc(Cl)cc2)=[NH+]/C1=N\\OC(C)=O', 'NS(=O)(=O)CC/C=C(\\C=C\\C=C/c1ccc(Br)cc1)NC(=O)c1ccccc1Cl', 'CN[C@H](c1cccc(OC)c1)c1cc(F)cc(F)c1', 'Oc1ccc(F)c(C[NH2+][C@H]2CCCc3ccccc32)c1', 'Clc1ccccc1-c1nnc(SCc2cccc(F)c2)n1C[C@H]1CCCO1', 'C=C[C@@H](OC(=O)Cc1cccc(Cl)c1)c1ccccc1', 'COC(=O)[C@@H](C)n1nc(C)c2c(-c3ccccc3)nccc21', 'ClCc1csc2ncncc12', 'N#Cc1c([N+](=O)[O-])c2cc([N+](=O)[O-])ccc2n1Cc1ccc(F)cc1', 'Cc1ccsc1-c1nnc(SCC(=O)[O-])s1', 'O=C(Cc1c(Cl)cccc1Cl)Nc1nnn[n-]1', 'Oc1cc(O[C@@H]2ON=C(c3ccccc3)C[C@H]2C(=O)[O-])ccc1O', 'N#C[C@@]1(Sc2ccccc2)NC(=O)C=C1c1ccccc1', 'Oc1ccc(F)c(F)c1/N=N/OCc1ccc(F)cc1', 'Clc1cc(Br)c(-c2ccccc2Cl)cc1Cl', 'COc1nc(CN(C)c2nc(C)nc3c2c(C)nn3-c2cccc(C)c2C)cs1', 'N[C@@H](COCc1ccc(F)cc1)c1cc(Cl)ccc1Cl', 'Clc1ccccc1[C@H]1[NH+]=c2ccccc2=[NH+]C1=O', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1C(=O)Nc1csc2ccccc12', 'Clc1cc(Br)c2nc(-c3ccccc3)[nH]c(=O)c2c1', 'Clc1ccc(-c2cc(OC[C@@H]3CCCCO3)co2)cc1', 'O=C(c1cn[nH]n1)N(c1c(Cl)cccc1Cl)N1CCSCC1', 'Clc1cccc2cc([C@H](Br)[C@@H]3COc4ccccc4O3)ccc12', 'Oc1ccccc1[C@@H]1C(c2cccs2)=CN=C1c1ccc([N+](=O)[O-])cc1', 'ClCc1cccc(Br)c1[N+](=O)[O-]', 'Clc1ccccc1Oc1cnc2snnc2c1', 'Clc1ccccc1Oc1cnc2sncc2c1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1O', 'Clc1ccc(Br)cc1S(=O)(=O)Nc1ccc(Cl)cc1', 'Clc1ccc(Br)cc1S(=O)(=O)Nc1ccc(Cl)c(F)c1', 'Cc1nnnn1CC(=O)N[C@@H](C)c1nc(C)cs1', 'Oc1ccc([N+](=O)Cc2coc3ccccc23)cc1OC(F)F', 'Clc1ccccc1OCc1csc(Br)c1', 'Clc1cc(Cl)c2c(c1)N(Cc1ccc(Cl)cc1)CCCC2', 'CC[C@@H](C)OC(=O)c1nc(C)cc(C(N)=O)n1', 'Clc1nnn(-c2ncccc2[N+](=O)[O-])c1I', 'Oc1ccc(F)c(F)c1NCCC(F)(F)F', 'Clc1ccc(-c2nonc2C2CC2)cc1F', 'Clc1ccccc1Cn1c[nH+]c(-c2cccc(Cl)c2)n1', 'Clc1ccccc1CSC1=NC2=CC=N[C@@H]2S1', 'Clc1cccc(F)c1N[C@@H]1CCSc2ccccc21', 'O=C(/N=C/c1ccc([N+](=O)[O-])cc1)NCCC(F)(F)F', 'Clc1ccc([C@@H](NN/c2ccc(Cl)cc2Cl)c2cc(F)ccc2F)cc1', 'O=[N+]([O-])c1cc(Cl)ccc1Oc1ncccn1', 'Clc1nnnn1-c1c(F)cccc1F', 'Clc1nnnn1-c1c(Cl)cccc1Cl', 'Clc1nnnn1-c1c(Cl)cccc1Cl', 'Oc1ccc([N+](=O)[O-])c(COc2ccc(C(F)(F)F)cc2)c1', 'Clc1ccccc1-c1csc(/N=C2\\CCCN2CC(F)(F)F)n1', 'Cc1c(-c2cccs2)ncnc1N1CCC([C@H]2CCN(C)C2=O)CC1', 'Clc1ccccc1O/C=COc1ccc([N+](=O)[O-])cc1', 'Clc1ccsc1[C@H]1Oc2ccccc2[C@@H]1Nc1cccc([N+](=O)[O-])c1', 'Oc1ccc(F)c(C[NH+]2CCC(Nc3ccc([N+](=O)[O-])cc3)CC2)c1', 'Clc1ccc(CNc2c[nH]c(C(=O)[O-])c2)o1', 'Oc1ccc(/C=C/N=C([O-])Oc2ccc([N+](=O)[O-])cc2)c(OCc2ccccc2)c1', 'COc1ncnc(Nc2cc(C)cc(C)c2)c1C', 'Clc1cc(/N=C\\c2csc(-c3ccc(F)cc3)n2)ccc1Br', 'Clc1ccccc1Oc1cc(Br)ccc1C(=O)[O-]', 'O=C(Cc1c(F)cccc1F)Oc1ccc2c(c1)OCCO2', 'Clc1ccc(C[NH+]2C[C@@H]3CCC[C@@H]3C2)cc1[N+](=O)[O-]', 'Oc1cccc2cc(O)ccc12', 'Oc1cccc2cc(O)ccc12', 'ClC(=S)N1Cc2ccccc2S[C@@H]1[C@@H](O)C1CCOCC1', 'O=[N+]([O-])c1c(O)ccc2c1NC(Cc1ccccc1Cl)C2=O', 'O=C(CS(=O)(=O)c1cnn(-c2ccccc2)c1)NCCc1c[nH]c2ccccc12', 'Clc1cccc(F)c1N/C=C/c1ccc([N+](=O)[O-])cc1', 'O=C(c1ccsc1)N1CCC(Oc2ccc(F)cc2)CC1', 'O=C(c1cn2ccsc2n1)[C@@H]1CCCCO1', 'OCc1nc(C(F)(F)F)cs1', 'Cc1ncccc1C(=O)Nc1cncc(C(F)(F)F)c1', 'COC(=O)C1(C#N)CC[C@H](n2ccc(=O)[nH]c2=O)CC1', 'O=C(C[NH+]1CC[C@@H]2OCC[C@@H]2C1)Nc1ccc(F)cc1F', 'Oc1ccccc1/N=N/c1ccc(C(F)(F)F)cn1', 'Clc1ccc(Br)cc1O[C@@H](CCC12CC=CCC2)C1', 'Clc1ccccc1-c1csc(N2CCOCC2)n1', 'Clc1cccc(/C=C2\\SC(=S)NC2=O)c1', 'O=C(c1cccc(S(=O)(=O)[N-]c2ccc(Cl)c(Cl)c2)c1)N1CCCCC1', 'Clc1ccsc1N[C@H](C1CCOCC1)S(=O)(=O)c1cccc2nonc12', 'Oc1ccccc1/C([O-])=N/S(=O)(=O)c1ccc(Br)cc1', 'Oc1cccc([C@H](c2[nH]nc3ncccc23)[NH+]2CCC(O)CC2)c1', 'Clc1cc(/N=C\\c2ccco2)c2ccccc2c1O', 'Oc1cccc2cc(/C=C/C(=O)N[C@@H]3OCCO3)ccc12', 'Oc1ccc(F)cc1N[C@H](CO)c1ccc(F)cc1', 'Clc1ccsc1SC/C=C/c1nc2sccn2c1Cl', 'O[C@@H]1c2ccccc2-c2nccnc2S1(=O)=O', 'Oc1ccccc1O[C@@H](c1ccccc1)c1cccc(F)c1', 'Nc1c(Oc2cc(Cl)ncn2)cnn1-c1ccc(F)cc1', 'ClCc1nonc1Sc1ccccc1', 'NS(=O)(=O)CCC[NH+](C(=O)c1ccc(Cl)s1)c1ccccc1', 'Oc1cccc(-c2ccc(C(=O)Nc3cccc(F)c3)nc2)c1', 'Clc1ccc(Br)cc1OC(=O)c1cccnc1', 'Clc1cc(F)c(Nc2ncccc2C(=O)N2CCCCC2)cc1', 'Oc1cc(Br)cc2c(Cl)cc(SCc3ccoc3)nc12', 'Oc1ccc(F)c(Cl)c1OC1CC[NH2+]CC1', 'Nc1[nH]ncc1N(C(=O)c1ccc(Cl)cc1N(C)Oc1ccccc1Cl)S(C)(=O)=O', 'Oc1ccc(Oc2cc(Cl)ccc2N2CCOCC2)cc1', 'O=[N+]([O-])c1cc(S(=O)(=O)CCCc2cn3ccccc3n2)ccc1F', 'CCN1C(=O)N=C(NNc2ccc(C)cc2C)C1=O', 'NS(=O)(=O)Cc1cnn(-c2ccccc2)c1', 'Clc1ccccc1Nc1nc2sccn2c1Br', 'Oc1cccc2cc(/C=N\\C(=O)c3ccc4c(c3)OCCO4)oc12', 'COc1nn(-c2ccccc2)c(C(C)C)c1NS(=O)(=O)c1cccc(C(F)(F)F)c1', 'O=[N+]([O-])c1ccc(OCc2ccc(C(=O)OCc3ccccc3)o2)cc1', 'O[C@@H](Sc1ccncc1)c1cccc(Br)c1', 'Oc1ccc([N+](=O)CCl)cc1OCC[NH+]1CCCCCC1', 'Clc1nnc(NCc2cccc(F)c2)n1-c1ccccc1', 'Clc1nncn1c1c(Cl)nc(Cl)cc1Cl', 'Clc1nnc(Br)cc1Oc1ncccn1', 'ClCc1csc(Sc2ccccc2)n1', 'O=[N+]([O-])c1ccncc1Sc1ccc(-n2cccn2)nc1', 'Clc1ccc(-c2csc3[nH]ccc23)cc1O', 'NS(=O)(=O)/C=C/CSc1ccccc1', 'Clc1ccccc1Oc1cc(Cl)c(Br)s1', 'O=[N+]([O-])c1c([O-])[nH]c(=O)[nH]1', 'Oc1ccc(F)c(OS(=O)(=O)c2ccc(I)cc2)c1', 'O=C(c1cc(Br)cnc1Br)N1CCN(C(=O)c2cccs2)CC1', 'Clc1ccccc1-c1csc(C(F)(F)F)n1', 'Clc1cccc(Br)c1O[C@H](C(=O)c1cccnc1)c1cccs1', 'Oc1ccc(O[C@@H]2CCC[NH2+]C2)cc1', 'Clc1ccc(Br)c(Cl)c1C[NH+]1CCC(O)CC1', 'Clc1ccccc1O/N=C(/OCc1cccc([N+](=O)[O-])c1)n1cncn1', 'Clc1ccc([C@@H](NC2COC2)C(F)(F)F)cc1', 'Oc1cccc(NCc2cccc(Cl)c2Cl)c1', 'Clc1ccc(Cc2cccc([N+](=O)[O-])c2)cc1', 'Clc1cc(/N=C/c2c(Cl)cccc2Cl)nc2ccccc12', 'O=[N+]([O-])c1c([N+](=O)[O-])cnc2[nH]cnc12', 'Cc1ncsc1CNC(=O)CCc1nc2c(Br)cccc2[nH]1', 'Oc1ccc([N+](=O)[O-])c(-n2ccnc2)c1C(F)(F)F', 'Clc1ccccc1-n1nnnc1SCc1cc(Cl)c2c(c1)OCO2', 'Clc1ccc(Br)cc1[C@@H](Cl)c1ccccc1', 'Clc1ccc(Br)cc1-c1ccccc1Cl', 'Clc1cccc(/N=C/c2cc([N+](=O)[O-])ccc2OC[C@@H]2CCCO2)c1', 'CCCCN(Cc1cccc(C#N)c1)C(=O)N[C@@H]1C[C@@H]1C', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n', 'Clc1ccc(Br)cc1Cl\n', 'Clc1ccc(Br)cc1Cl', 'Clc1ccc(Br)cc1Cl', 'Clc1cc(Cl)ccc1Cl', 'Clc1cc(Br)ccc1Cl', 'Cc1snc(C)c1S(=O)(=O)N[C@@H](CO)c1cccc(Cl)c1', 'Clc1cccc(Br)c1C[N+]1=CCCC1', 'Clc1ccc(-c2cccnc2)cc1N1CCOCC1', 'Oc1cc(-n2cc[nH+]cc2=O)cc(I)c1', 'Clc1ccccc1-c1nccn1C1CCN(S(=O)(=O)c2cccc(Cl)c2F)CC1', 'Clc1ccccc1Oc1cc(Br)c(Cl)cc1Cl', 'Clc1ccc(Br)cc1/N=C/c1ccc(F)cc1', 'O=[N+]([O-])c1c(F)cc([N+](=O)[O-])cc1OCCCCO', 'CNS(=O)(=O)c1ccc([N+](=O)[O-])cc1N', 'Oc1ccc([N+](=O)COc2ccccc2C(=O)[O-])cc1', 'Oc1cccc2ncnc(Nc3cccc(I)c3)c12', 'Clc1cc(C2OCCC2)ccc1-n1cnnn1', 'Oc1cc(F)nc([C@@H]2CCCN2c2ncccn2)c1', 'Clc1cccc(NCC(=O)Nc2ccncc2C(F)(F)F)c1', 'Oc1cccc2cc(OCc3ccc(F)cc3)cnc12', 'O=[N+]([O-])c1c([O-])n2ncc(C3CC3)c2ncn1-c1ccc(F)cc1F', 'Oc1ccc(F)c([C@@H](C[NH2+]C(N)=O)[NH+]2CCCC2)c1', 'O=C(/N=C/NCC1CCCC1)c1ccc(F)cc1F', 'Oc1cc(F)nc(Cl)n1', 'Clc1cc(Cl)nn1/N=C\\c1ccccc1Cl', 'Clc1cc(Br)c2ncnn2c1Cl', 'Clc1ccc(-c2nn(-c3ccc(F)cc3)c3c2NCCC[C@H]3OC(F)(F)F)cc1', 'Oc1cccc(N2CC[NH+](Cc3ccc(Cl)o3)CC2)c1', 'O=[N+]([O-])c1cc(Nc2ccccc2Cl)c2ccc(F)cc2n1', 'O[C@@H]1c2cc(S(=O)(=O)[O-])ccc2[C@H]1c1ccccc1', 'Clc1ccc(Br)cc1-n1ncc(Br)c1C(=O)[O-]', 'O=C(COc1cc(Br)c(Cl)cc1)NCc1cccc([N+](=O)[O-])c1', 'O=C(CCc1cc2ccccc2oc1=O)OCc1cccnc1', 'Oc1cc(F)ccc1[C@@H]1CCCCN1C(=O)c1cc([N+](=O)[O-])cs1', 'Oc1ccc([N+](=O)[O-])c(Sc2cc(F)ccc2F)c1', 'Clc1ccc(-c2csc3s/cnc23)cc1O', 'Oc1ccc(F)cc1CN[C@@H](C1CC[NH2+]CC1)c1ccc(F)cc1', 'ClCc1cccc(Sc2nncn2C2CC2)c1', 'O=[N+]([O-])c1ccc2oncc2c1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1', 'O=[N+]([O-])c1ccc2[nH]nc(-c3ccccc3)c2c1', 'Nc1n[nH]c2cc([N+](=O)[O-])ccc12', 'Oc1cc(O[C@@H]2CCCc3ccccc32)c2ccccc2c1[O-]', 'Oc1cc(F)c([C@@H](O)CN2CCCCCC2)c(Cl)c1', 'Clc1ccccc1[C@H](Cl)c1ccc(Cl)cc1F', 'O=[N+]([O-])c1ccncc1-c1ccc2c(c1)OCO2', 'CNC(=O)C(=O)NC[C@@](C)(O)c1ccc(F)cc1', 'O=[N+]([O-])c1c(Br)cccc1N1CCOCC1', 'O=C(CCS(=O)(=O)c1cccc(C(F)(F)F)c1)NC[C@H](O)c1ccccc1', 'Clc1ccsc1Nc1nc(-c2ccc(Br)s2)no1', 'O=C(c1cn(-c2ncccc2Cl)o1)N1CCOCC1', 'Clc1ccccc1Oc1cc(Br)c([N+](=O)[O-])cc1Br', 'Clc1cccc2nc(SC[C@H]3CSCCO3)nn12', 'CN=C(C#N)CC1(c2ccc(OC)cc2)CC1', 'O=[N+]([O-])c1cc(N2CCOCC2)cs1', 'Clc1ccccc1OCc1csc2ccc(F)cc12', 'Clc1ccccc1OCc1csc(Cl)c1', 'O=C(CSCc1n[nH]c2ccccc12)n1cccn1', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(F)cc1NCc1ccco1', 'CCn1nc(C)c(CNC(=O)c2cccc(-n3cccc3)c2)c1C', 'Clc1cccc(F)c1N1CCOc2cccc(F)c21', 'ClCc1cccc([N+](=O)[O-])c1', 'NS(=O)(=O)C[C@H]([NH3+])c1cccs1', 'Clc1cc(Br)c2nc([C@H]3CCC[NH2+]C3)sc2c1', 'Clc1nnnc(CI)c1', 'Oc1cc(COc2ncccc2Cl)ccc1F', 'Clc1cc(Cl)ccc1Cn1cc(C(F)(F)F)ccc1=O', 'Clc1nnn(-c2cccc(Cl)c2)c1Cc1ccccc1Cl', 'Oc1cc(Cl)cnc1SCc1ccc(F)cc1', 'O=[N+]([O-])c1cc(Cl)c(OCC(F)(F)F)cc1Br', 'Clc1nnnc(CSc2nncn2-c2ccccc2)n1', 'Cc1nnsc1CN1CCN(C(=O)c2ccc(Cl)cc2)CC1', 'Clc1ccc(Br)c(F)c1Cn1c[nH+]c(-c2cccc(Cl)c2)n1', 'Oc1cc(F)nc(Sc2cc(Br)ccc2O)n1', 'O=[N+]([O-])c1ccc(CNC[C@@H]2CCC[NH2+]2)cc1', 'Clc1ccc(C[NH2+]C[C@H]2CCCc3ccc(F)cc32)o1', 'O=C(/N=c1/[nH]cccc1Nc1ncccn1)c1cccs1', 'CN(c1cccc(N)c1)S(=O)(=O)c1cccc([N+](=O)[O-])c1', 'Clc1cc(/C=N\\Oc2ccccc2)sc1N1CCCC1', 'Clc1cc(/C=N/n2cccc2)nc(C(=O)[O-])c1', 'O[C@@H]1C[C@@H](c2cccs2)N(c2ccc(Cl)cc2)C(=O)O1', 'Clc1ccccc1N1N=C(N2CCN(c3ccccc3)CC2)SC1=S', 'Nc1cn[nH]c(=S)n1', 'Clc1ccc(Br)cc1-c1ccccc1', 'Clc1ccc(Br)cc1-c1ccccc1', 'Oc1cc(F)ncc1C(=O)NNc1ncc(Cl)c(Cl)c1Cl', 'Oc1cc(-n2cc(F)cn2)cc(C(=O)Nc2ccc(F)cc2)c1', 'N#C[C@@H]1C(=O)NCCN1CC1CC[NH+](Cc2ccccc2)CC1', 'Clc1nnc(Cl)n2ncc(-c3ccc(F)cc3F)c12', 'Clc1nnns1\n\n\n\n\n\n\n\n', 'Clc1nnns1\n\n\n\n\n\n\n', 'Clc1nnns1\n\n\n\n\n\n', 'Clc1nnns1\n\n\n\n\n', 'Clc1nnns1\n\n\n\n', 'Clc1nnns1\n\n\n', 'Clc1nnns1\n\n', 'Clc1nnns1\n', 'Clc1nnns1', 'Clc1nnns1', 'Clc1nnns1', 'Clc1ccsc1Sc1nnc(-c2ccccc2)n1C1CCCC1', 'Clc1ccccc1Oc1cc(Sc2nc3c(s2)CCCC3)cs1', 'Oc1cc(-n2nc([O-])[nH]c2=O)c(=O)n(Cc2cccc(Cl)c2)c1', 'Oc1cc(-n2cnc(Sc3nc(C4CC4)ns3)c2[O-])ccc1O', 'Clc1cccc([S@@]Cc2nc(-c3ccccc3)no2)c1', 'Clc1ccccc1OCc1cccc(C(F)(F)F)c1', 'Clc1ccc(-c2nn(-c3ccc([N+](=O)[O-])cc3)c(Cl)c2Cl)cc1', 'Oc1ccccc1-n1nnnc1SCc1ccc(Cl)cc1Cl', 'Oc1ccccc1-n1nnnc1SCc1ccc(Cl)c(Cl)c1', 'Clc1ccc(Br)cc1NC(=S)NCc1ncc[nH]1', 'Oc1cccc2cc(OC(F)F)oc12', 'Cc1ccsc1/C=C/C(=O)N1CCC(C(=O)[O-])CC1', 'Clc1ccc(Br)cc1/N=N\\C(=C\\c1nc2c(Cl)cccc2s1)C(=O)[O-]', 'Clc1nnnn1Cc1csc2ccccc12', 'Oc1ccccc1OC(=O)CN1CCc2cc(F)ccc2C1', 'Oc1ccc(F)cc1Nc1cccc(C(=O)N2CCOCC2)c1', 'Clc1nnnn1-c1cn(Br)cn1', 'Clc1nnc(Br)c(Br)c1C(=O)[O-]', 'Oc1cc(Cn2ccc3cc(Cl)ccc32)c([N+](=O)[O-])ccc1=O', 'C=CCCC[C@@H](C)OC(=O)c1ccc2c(c1)OCO2', 'COc1nn(-c2cccc(C(N)=O)c2)c(C(=O)[O-])c1C', 'O=[N+]([O-])c1cc(Cl)nc(Nc2ccc(Br)cc2F)c1', 'O=[N+]([O-])c1cc(OC[C@@H]2CCCO2)cs1', 'C[C@@H](CC(=O)Nc1ccc2c(c1)OCO2)CCC(C)(C)C#N', 'Clc1nnnn1-c1cc(Br)cc(Br)c1', 'Oc1cc(-n2nc(Cl)cc2SCC(F)F)nc2ccccc12', 'Nc1nc(-c2cccs2)nc2scnc12', 'Oc1ccccc1[C@H]1OP(=O)OCCCN1C(=O)c1ccccc1', 'Clc1cc(Br)cnc1Oc1ccccc1[N+](=O)[O-]', 'Clc1ccc(Br)c(-n2nnc(C(F)(F)F)c2C2CC2)c1', 'O=C(CN(c1cnn(C(F)F)c1)S(=O)(=O)c1ccccc1)N1CCc2ccc(Cl)cc2C1', 'Oc1cc(O)c([N+](=O)[O-])cc1OCCCC(F)(F)F', 'O=C(c1cn2ccsc2n1)N1CCC[C@@H]1c1cccc(Cl)c1', 'O=C([O-])/C=C/[C@H]1Cc2ccccc2O1', 'Clc1cc(/C=N/Sc2cnn(-c3ccccc3)c2Br)ccc1[N+](=O)[O-]', 'Oc1ccc(F)cc1NS(=O)(=O)c1ccccc1', 'Clc1ccc(-c2nnnn2-c2cc([N+](=O)[O-])ccc2F)cc1', 'Cc1cnc2ncc(C(=O)OCC#C/C=C/c3ccc(C#N)c(Br)c3)cc2c1', 'O=[N+]([O-])c1cc(Nc2cc([N+](=O)[O-])c(Cl)cc2)ccc1Br', 'O=C(/C=C\\Nc1cccc(Br)c1)c1ccc(F)cc1F', 'ClC([O-])(Br)Cc1cccc(Cl)c1Cl', 'Clc1ccsc1[C@H]1n2cccc2C2=NC1=NC2', 'Clc1ccc(-c2nn(-c3cc(C(F)(F)F)ccc3Br)[nH]c2=O)cc1', 'Cc1ncsc1C(=O)N1CCN(C[C@H](O)C(C)(C)CCC#N)CC1', 'Clc1ccc(-c2cc[nH]n2)c2ncccc12', 'Clc1cc(Br)cn1Nc1ncccc1F', 'Clc1cc(NSc2nc(-c3cccs3)nc3c2CCCC3)ccn1', 'OC[C@H](CO)Nc1ccc(F)c(Cl)c1', 'Nc1nonc1COc1cccc(-c2ccco2)n1', 'O=[N+]([O-])c1c(Cl)ncc(Cl)c1Cl', 'O=[N+]([O-])c1c(Br)scc1C(=O)Nc1cccs1', 'Clc1ccsc1S(=O)(=O)c1cc(Cl)cc(Cl)c1', 'O=[N+]([O-])c1c(Cl)nsc1-c1ccccc1', 'Clc1ccsc1-c1nc(Cl)ncn1', 'Clc1ccccc1OCc1cc([N+](=O)[O-])ccc1Br', 'N#Cc1n[nH]c(=O)c2cc(Cl)cc(-c3ccc(F)cc3F)c12', 'NS(=O)(=O)/C(=N\\[N+](=O)[O-])C(=O)N1c2ccccc2S[C@H]1C(C)C', 'Clc1cc(Br)c2nc(O)ncc2c1O', 'Clc1cc(Br)c2nc(O)ncc2c1O', 'Clc1ccccc1Oc1cc(SCC(=O)[O-])cc(Cl)n1', 'Cc1sc(NC(=O)C(C)(C)C)cc1C(=O)N(C)CCC(F)(F)F', 'Clc1nnnn1c1cc(F)ccc1F', 'Clc1nnnn1c1cc(Cl)c(Cl)cc1Cl', 'Clc1nnnn1-c1ccc(Br)cc1F', 'Clc1nnnn1-c1cc(F)ccc1F', 'Clc1nnnn1-c1cc(F)ccc1Cl', 'Clc1nnnn1-c1cc(F)ccc1Br', 'Clc1nnnn1-c1cc(F)cc(F)c1F', 'Clc1nnnn1-c1cc(Cl)ccc1Cl', 'Cc1ccsc1OCC(=O)Nc1cccc(C(=O)N2CCOCC2)c1', 'O=C(c1cnnn1-c1ccccc1)N1CCC[C@H](c2ncccc2Br)C1', 'C[C@@H](c1ccoc1)n1c(N)nc2cccnc21', 'Nc1[nH]nc(-c2cccc(Cl)c2)c1N', 'Clc1nncn1-c1cccc(Cl)c1', 'Clc1nncc(-c2cccc(F)c2)n1', 'Clc1ccccc1Oc1cnc2sc(C3CC3)nc2c1', 'O=C(/C(CO)[C@@H]1CCC[C@@H](C(F)(F)F)C1)c1ccc([N+](=O)[O-])cc1', 'Oc1ccc(/N=c2/[nH]c(-c3cccc(Cl)c3)n[nH]2)c(Br)c1', 'ClCc1nnsc1[N-][C@@H]1CCS(=O)(=O)C1', 'Oc1cccc(C(=O)Nc2ccccc2-c2ccco2)c1', 'Clc1cc(Br)c(Nc2nc([O-])c3ccccc3n2)c(O)c1', 'O=C([C@H]1c2cc(F)ccc2OC1)N1CCN(c2ccc(C(F)(F)F)cn2)CC1', 'Oc1cc(F)ccc1CN1CCN(S(=O)(=O)c2cccc(OCCn3cncn3)c2)CC1', 'Clc1ccsc1C(F)(F)F', 'Oc1cc(C=NSCc2ccsc2)nc2sccc12', 'Clc1cccc2c(Oc3cccc(Cl)c3)n[nH]c12', 'Clc1ccccc1-c1nnnn1Cc1ccc(F)cc1F', 'Clc1ccsc1S(=O)(=O)N1CCN(c2ccccc2)CC1', 'Clc1ccsc1C[N-]Oc1ccc2c(c1)CCC2', 'Oc1ccc(Oc2nc(Br)c3sccc3n2)cc1', 'Clc1cccc(Br)c1NC1=NS(=O)(=O)CC=CC1', 'Clc1ccccc1Cc1ccc(Cl)c(F)c1', 'Clc1ccc(Br)cc1-c1ccccc1F', 'Clc1cc(Br)ccc1-c1ccccc1F', 'Oc1ccc([N+](=O)C2CCCC2)cc1OC[C@@H]1CCC[C@@H]1O', 'Oc1ccc(F)cc1OCc1ccn(-c2cccc(F)c2)n1', 'O=C(c1ccccn1)Nc1cccc(NC(=O)c2ccco2)c1', 'O=[N+]([O-])c1cc(Br)c2[nH]cnc2n1', 'Clc1ccccc1Oc1cc(Cl)c(CO)cc1', 'O=C(c1cc([N+](=O)[O-])on1)N1CCC[C@@H]1c1ccccc1', 'Oc1cc(Br)cnc1N[C@@H]1CC[NH2+]C1', 'O=C(c1c[nH]nc1-c1ccccc1)N1CCSc2ccccc21', 'OCS[C@@H]1N=c2ccccc2=[NH+]C1=O', 'Clc1cccc([S@@](=O)c2nnc(N3CCCC3)o2)c1', 'Oc1cc(-n2cnc(Br)n2)ccc1N1CCOCC1', 'Nc1[nH]nc2nc(SCc3ccccc3C(F)(F)F)cnc12', 'CCc1[nH]c(CCl)c(C#N)c1N', 'O=C(/C=C1CC(C)(C)[NH+](Cc2ccccc2)C1=O)c1cc(Cl)c(Cl)cc1Cl', 'O=[N+]([O-])c1cc(OC2CCCC2)c2ccccc2c1F', 'Oc1ccccc1OSCc1n[nH]c2nc([O-])oc12', 'ClCc1n[nH]c2ccc(C(F)(F)F)cc12', 'ClCc1csc2nc(C(F)(F)F)cn12', 'Clc1ccc(Br)cc1/N=N\\Nc1cccc(C(=O)[O-])c1', 'Clc1cc(/C=C(/[O-])/N=[N+]=NC(=O)c2ccccc2)cc(Br)c1O', 'N#C[C@@H](O)C[C@@H]1COCCN1Cc1ccc(Br)cc1', 'Clc1ccccc1C(=O)OCC(F)(F)F', 'N[C@@H](Nc1ccc(F)c(Br)c1)c1ccccc1', 'O=C([O-])[C@@H]([NH3+])c1csc(Cc2cccc(F)c2)n1', 'Oc1ccccc1N1C(=O)c2ccccc2C1=O', 'O=C(c1cnn2cccnc12)N1CCC[C@H]1c1cccs1', 'NS(=O)(=O)NC[C@](O)(c1cccs1)C(Cl)(Cl)Cl', 'Oc1ccccc1[C@@H]1[NH+]=c2ccccc2=[NH+][C@@H]1c1cccc(Cl)c1', 'ClCc1cn(N[C@@H]2CCOC3(CCOCC3)C2)s1', 'Clc1ccc(-c2cccnc2Sc2nnc3ccccn23)cc1', 'Oc1ccc(/C=C(/SCc2cccc(C#N)c2)NN[C@H]2CCCO2)c([N+](=O)[O-])c1', 'Clc1nnnn1c1c(F)cc(Oc2ccc(F)cc2)c(Cl)c1O', 'Nc1nonc1/N=N/c1ccc(-c2ccccc2Cl)c(Cl)c1', 'Clc1ccccc1Oc1cnc2cc([N+](=O)[O-])cnc2n1', 'Oc1cccc(Nc2cccc(F)c2Br)c1O', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(O)c(CN2CCCCC2)c1Cl', 'Clc1cccc(F)c1C(=O)Nc1ccc(-c2cscn2)cc1', 'Oc1cc(-n2nnnc2C2CC2)c[nH]1', 'OC[C@H]1NC(=S)c2cccnc2C1=C(c1cccs1)N1CCCC[C@H]1Cc1ccccc1', 'Oc1ccc([N+](=O)[O-])c(NCc2ccccc2)c1', 'CNC(=O)CCNC(=O)c1ccc(F)cc1F', 'O=[N+]([O-])c1c(F)ccc2ncn([C@@H]3CC(=O)Oc4ccccc4N3)c12', 'Clc1cc(/C=Cc2sncn2)no1', 'Oc1cc(Br)ccc1Oc1ncc(Br)c(O)c1F', 'Clc1ccccc1O/N=C(/Cl)c1nccs1', 'Oc1cc(F)cc2[nH]ncc2c1=O', 'OCSc1nc2ccccc2cc1NCc1cccc(Br)c1', 'Oc1cccc(Br)c1C[N+](=O)[O-]', 'Clc1cccc(NCC[NH2+]Cc2sccc2Cl)c1', 'O=[N+]([O-])c1c(F)cc([O-])cn1', 'O=C(c1c[nH]c(Nc2cccc(F)c2)n1)N1CCC[C@@H](C(F)(F)F)C1', 'Oc1cccc(N2CC[C@H](NCc3ccc(F)cc3F)C2)c1', 'Clc1ccc(Br)cc1CC/N=c1\\sc(-c2cccc(Cl)c2)n[nH]1', 'N[C@@H](N1CCN(c2ccc(F)cc2)CC1)c1cc(F)cc(Br)c1', 'Clc1ccsc1[C@@H]1[C@H]2C(=O)N(c3cccc([N+](=O)[O-])c3)CC2=N1', 'ClCc1cc(Oc2c(Cl)cccc2Cl)ncn1', 'Clc1cccc(CCOC(F)(F)F)c1', 'O=[N+]([O-])c1cc(F)cc(CNc2ccccc2F)c1', 'NS(=O)(=O)/N=C(\\[O-])c1n[nH]c(-c2ccoc2)n1', 'Clc1cc(Cl)c2sc(N[C@H]3CCS(=O)(=O)C3)nc2c1', 'Clc1ccc([C@@H](Br)[C@@]23CC[C@H](C[C@@H]2C)C3=O)cc1Cl', 'NS(=O)(=O)Nc1cc(-c2ccco2)nc2ccc(Cl)cc12', 'Oc1ccc([N+](=O)[O-])c(NN=C2CCCCC2)c1', 'Oc1cc(C[NH+]2CCCC2)cc([N+](=O)[O-])c1O', 'ClC[C@@H]1Oc2ccccc2C(=O)N1c1ccccc1C(F)(F)F', 'Clc1ccsc1NC(=S)Nc1ccc2c3c(cccc13)CC2', 'O=[N+]([O-])c1ccc2[nH]ncc2c1/N=N/c1csc(Br)c1F', 'Clc1ccc(Br)cc1/C=N\\C(=O)C1=NN=C1Cc1ccccc1', 'O=C(Cc1cnccn1)Nc1cc(Cl)c(Cl)cc1Cl', 'O=C(c1c[nH]nc1-c1cc(Cl)ccc1Cl)N1CCOCC1', 'O=C(CSCCNc1ccc2ccccc2n1)Nc1ccccc1OC(F)F', 'Oc1cccc2ncn(/C(CO)c3nc(-c4ccccc4F)n[nH]3)c12', 'Oc1cccc([C@H](NC(=O)Cn2cncn2)c2cccs2)c1', 'O=C([O-])c1[nH]nc(-c2ccc([N+](=O)[O-])cc2)c1Br', 'O=C(/N=C\\N1CCN(c2c(C#N)cc(F)cc2F)CC1)c1ccccc1', 'Clc1cccc(Nc2cc(F)cc(Br)c2)c1', 'Clc1cccc(Nc2cc(Br)ccc2F)c1', 'Clc1ccc(-c2cc(F)c([C@@H]3CCCN(c4nccs4)c3)CCN2)cc1', 'COc1nnnc(Sc2nnc(C)c(C)n2)c1C', 'Clc1cccc([S@@]c2nc3ccccc3c(=O)n2C[C@H]2OCCO2)c1', 'Oc1cc(C(F)(F)F)ccc1NCc1ccc(N2CCCC2)cc1', 'Clc1ccc(-c2cccnc2Sc2ccc(C[NH+]3CCCC3)cc2)cc1', 'Cc1nonc1Cc1nc(C)c(C)s1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1Cl', 'Clc1ccc([C@@H](N[C@@H](O)CO)C(F)(F)F)cc1', 'Clc1nnnn1Cc1cc(Cl)cc(Cl)c1', 'Clc1nnnn1Cc1cc(Br)ccc1O', 'ClCc1cc(Br)ccc1Oc1ccccc1', 'Clc1ccc(-c2n[nH]cc2-c2ccc(S(=O)(=O)C(F)(F)F)cc2)cc1', 'Clc1ccc([C@@H]([O-])Cc2ccc([N+](=O)[O-])cc2)cc1', 'N=Cc1cc([N+](=O)[O-])c(Cl)cc1Oc1ccccc1Cl', 'Clc1ccc(-c2cc([N+](=O)[O-])nc(N3CCOCC3)n2)cn1', 'Clc1cccc2ccc(/C=C3\\SC(=S)N(c4nonn4)C3)cc12', 'Oc1cccc(-c2cnc3cc(Cl)ccn3c2=O)c1', 'Oc1cccc(N2CCOCC2)c1', 'Oc1ccc(F)cc1[C@H]1Oc2ncccc2Oc2ccccc21', 'O[C@@H](c1cccs1)[C@H]1COc2ccc(F)cc21', 'Oc1ccc(F)c(-c2csc(Cc3ccc(F)cc3)n2)c1', 'ClC(/N=C1/N=C[C@@H]2c3ccccc3C(=O)N12)[C@@H]1COc2ccccc21', 'Clc1ccccc1Oc1cnc(F)c(Cl)c1', 'Clc1ccccc1O/N=C(/c1ccncc1)S(=O)(=O)c1cccc(Cl)c1Cl', 'O=[N+]([O-])c1ccc(CN2CCN(c3ncnc4scnc34)CC2)cc1', 'Nn1c(=O)oc(=O)n2ncn2c1=O', 'Clc1cccc(NCCNC(=O)c2cccc(F)c2F)c1', 'Clc1ccccc1COC(=O)[C@H]1Sc2ccccc2S1(=O)=O', 'Clc1ccccc1Cn1nc2cccnc2c1CSCCO', 'O=[N+]([O-])c1cc(F)c2cccnc2c1Oc1ccccc1', 'Nc1[nH]nc(NNC(=O)[C@@H]2COc3ccc(Br)cc32)n1', 'CCO[P@](C)(=O)c1ccc(Cl)c([N+](=O)[O-])c1', 'O=[N+]([O-])c1c(Br)cncc1Sc1ccc(C[NH3+])s1', 'Clc1cccc2nn[nH]c1c1cccc2cc(Cl)c1F', 'Clc1cccc([S@@]c2nnnn2-c2ccccc2)c1', 'Clc1nnnc(SCn2ncc3ccc4c(c23)CCCCCC4)c1', 'Clc1nncnc1N=C\\c1ccc(Cl)cc1Cl', 'Clc1ccc(Cc2ncc(C(F)(F)F)s2)cc1', 'Clc1ccc(Cc2nc(C(F)(F)F)cs2)cc1', 'O=[N+]([O-])c1ccc(/N=C\\c2nc(Nc3cccc(F)c3)c(=O)[nH]c2[O-])cc1', 'NS(=O)(=O)/N=c1\\sc(-c2ccc(F)cc2)cc(-c2ccco2)n1', 'Cc1snnc1C(=O)N1CCc2cc(O)ccc2[C@H]1c1cccc(O)c1', 'O=[N+]([O-])c1ccc(S[C@@H]2CCCS[C@@H]2c2cccs2)cc1', 'O=C(c1cc2ccccc2oc1=O)N1CC[C@@H](n2cncn2)C1', 'Clc1cccc([S@@](=O)Cc2c[nH]nc2-c2ccccc2)c1', 'CN(c1cccc(Cl)c1)S(=O)(=O)c1c(C(N)=O)cnc2ccccc2c1=O', 'Clc1ccsc1[C@@H](Cl)Cc1cc(Cl)ccc1F', 'O=[N+]([O-])c1ccc(Sc2ccc(Cl)cc2)cc1', 'COc1nnn(C)c1C[C@H](C)OC(=O)c1cc(C)n(C)n1', 'O=C(c1cn2ccsc2c1Cl)c1ccc(Cl)c(Cl)c1', 'O=C(/N=C\\c1ccc(Cl)cc1)N1CCc2cc(Cl)cc([N+](=O)[O-])c2C1', 'Oc1cc(O)c2ccc(Cl)cc2c1OC(F)c1ccccc1', 'C[C@@H](OC[C@H]1CCCCO1)C(=O)Nc1ccc([N+](=O)[O-])cc1', 'Oc1cc(O)ccc1OCC(=O)NCC(=O)c1ccccc1O', 'Nc1nc([C@H]2CCCc3cccnc3N2)no1', 'Oc1cc(-n2nc(C(F)(F)F)cc2Cn2ccccc2=O)ccc1O', 'Clc1cc(Br)cc(/C=N/N2CCOCC2)c1', 'Clc1ccc(Br)cc1O[C@H](CO)C(=O)[O-]', 'O=[N+]([O-])c1cc([C@H]2CCCN(c3ncccn3)C2)c(F)c(Cl)c1', 'NS(=O)(=O)/C=C/Cc1ccccc1', 'Oc1cccc(Oc2cnnc(Nc3ccccc3)n2)c1', 'CCSC1=NN(c2ncccc2[N+](=O)[O-])C(=O)[C@@H]1C', 'Clc1cccc2cc(/C=N/c3cc([N+](=O)[O-])ccc3Cl)[nH]c12', 'Clc1ccsc1-c1noc(C(=O)Oc2ccc(F)cc2)n1', 'Clc1cccc2[nH]nc(SCc3ccsc3)c12', 'Clc1cccc2[nH]nc(SCc3ccsc3)c12', 'COc1nnnn1-c1ccc2nnnn2c1', 'O=[N+]([O-])c1ccnc2ccc(C(=O)N3CCC[C@@H](O)C3)cc12', 'Oc1ccc(F)cc1NS(=O)(=O)c1ccc(Cl)cc1', 'NS(=O)(=O)/C=C(c1cccs1)S(=O)(=O)Cc1ccccc1', 'Oc1cc(F)ncc1[C@H](O)c1cccc(F)c1', 'Clc1cccc(/N=N\\Nc2ncccn2)c1', 'Clc1ccsc1Sc1ccccc1OC(F)F', 'Cc1cc(C(=O)N[C@@H](C)c2ccc(Cl)cc2)c2ncnn2c1', 'OC1=C[C@H](Sc2c[nH]c3cccc([N+](=O)[O-])c23)C([O-])=N1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1[N+](=O)[O-]', 'O=[N+]([O-])c1cc(Cl)n(-c2ccc(F)cc2)n1', 'O=[N+]([O-])c1cc(Cl)n(-c2ccccc2F)n1', 'Clc1ccc(Br)cc1SC[C@H]1Cc2ncccc2-c2ccccc21', 'Clc1nnnc(Oc2cncc([N+](=O)[O-])c2)n1', 'Clc1cc(Nc2c([N+](=O)[O-])cc(Cl)cn2)ccc1F', 'Clc1cc(Cl)nc([O-])c1F', 'Clc1cc(F)c([N+](=O)[O-])cc1OC[C@@H]1CCCO1', 'O=C(/C=C\\c1nc2sccn2c1Cl)N[C@@H]1CCOC1=O', 'ClCc1csc2nc3c(cc12)CNC3', 'Clc1cc(Br)cnc1OC[C@H]1Cc2ccccc2S1', 'Clc1nnn(-c2nccs2)n1', 'NS(=O)(=O)C[C@H]1CCCC[NH+]1Cc1cccnc1', 'Clc1cccc2cc(-c3noc(C4CC4)n3)nnc12', 'Clc1ccc(Br)cc1[C@@H]1CCCN(C(=O)Nc2ccc3c(c2)OCO3)C1', 'ClCc1csc2nc[nH]c12', 'Clc1ccccc1Oc1cc(-c2cccs2)n[nH]1', 'Oc1cc(-n2cnnn2)c(NC2CC2)c(Cl)c1', 'Oc1cc(F)ncc1S[C@H](C(=O)[O-])c1ccccc1', 'ClC[C@H](O)c1ccc(Cl)cc1', 'Clc1ccc(Br)cc1S(=O)([O-])Br', 'Clc1ccc(Br)cc1S(=O)(Br)F', 'O=C(/C(c1cccs1)(Br)F)C(F)(F)F', 'CNC(=O)N1C[C@@H](c2ccccc2F)OC1=O', 'ClCc1csc2nc[nH]c(=O)c12', 'Clc1ccc(Br)cc1-c1cc2ccccc2cc1F', 'O=C(/N=C/c1ccc(Cl)c2ncccc12)N1CCN(S(=O)(=O)c2ccccc2)CC1', 'ClCc1nc(-c2ccc([O-])cn2)cs1', 'CN[C@H](CO)c1cccc([N+](=O)[O-])c1', 'Oc1cccc(Br)c1Cl', 'Clc1cc(/C=C2\\CCc3ccccc32)ccc1O', 'O=[N+]([O-])c1ccncc1N1CCSc2ccccc21', 'Clc1ccsc1-c1nnc(N2CCC(O)CC2)o1', 'O=C(Cc1cc2occc2[nH]1)NNC(=O)c1cc(Cl)ccc1Cl', 'Clc1nnn(Cc2ccn[nH]2)n1', 'N[C@@H]1[NH2+]N2CC(c3ccccc3)=N[C@H]2C1', 'Clc1ccccc1Nc1cc(F)cc(Br)c1', 'Oc1ccccc1[C@@H]1C[C@H](O)C[NH+]1Cc1ccc([N+](=O)[O-])cc1C(F)(F)F', 'O=C(/C=C\\C1(C(=O)Nc2cncc(-c3ccccc3)n2)CC1)N(C1CCCC1)S(C)c1cccnc1', 'Clc1ccc(-c2nn(-c3cc(F)ccc3F)c(=S)s2)cc1', 'Oc1cc(F)ncc1N[C@H]1CCCn2ccnc21', 'Clc1ccccc1Cn1nnnc1SCn1cccn1', 'O=C(CSCc1nnc2n1CCC2)NC1[C@@H](C#N)CCCc2ccccc21', 'Oc1ccc(F)cc1[C@H]1CCCN1c1ccccc1', 'COCCn1c(S[C@H](C(N)=O)C(=O)Nc2ccc(Cl)cc2)c(-c2ccccc2)c(C)n1', 'Clc1ccccc1[C@H]([NH3+])c1csc(Br)c1', 'OCc1n[nH]c(=O)c2cc(S(=O)(=O)N3CCCC3)ccc12', 'O=[N+]([O-])c1ccncc1-c1ccc([N+](=O)[O-])cc1', 'Oc1ccc([N+](=O)CCl)cc1N1CCCC1', 'O=[N+]([O-])c1ccncc1Cc1cccc(F)c1', 'Clc1cccc(Nc2cc(Cl)ccc2Cl)c1', 'ClCc1csc(Nc2ccc(OC(F)F)cc2)c1', 'Oc1ccc([N+](=O)c2ccc([N+](=O)[O-])cc2)cc1Oc1cccc2c1OCCO2', 'Clc1cc(Cl)nc(-c2cccnc2)n1', 'Clc1ccccc1[C@H](Nc1cccc(F)c1)C(F)(F)F', 'O=C(CS(=O)(=O)c1ccccc1Cl)Nc1cc(Cl)cc(Br)c1', 'O[C@@H]1c2cc(F)ccc2[C@H]2CCCC[C@@H]12', 'Oc1cccc2cc(/C=N/N=C\\c3ccco3)oc12', 'Oc1ccccc1Cn1ccnc1Sc1ncnc([O-])c1', 'Clc1ccc(-c2cccnc2SC[C@@H]2CCCO2)cc1', 'OCc1nn(S(=O)(=O)c2ccccc2)c2ccccc12', 'O=C(C[NH+]1CCOc2ncccc2C1)Nc1ccc(Cl)c(F)c1', 'O=[N+]([O-])c1cccc2ncn(Cc3ccccc3)c12', 'Oc1cccc(-c2c[nH]nc2[C@H]2CC(c3ccccc3)=NS2(=O)=O)c1', 'Clc1ccccc1Oc1cc(F)c(Br)cc1', 'Clc1ccccc1Oc1cc(Br)ccc1F', 'Clc1ccccc1Oc1cc(Br)ccc1F', 'Clc1ccccc1Oc1cc(Br)cc(F)c1', 'Clc1ccccc1NS(=O)(=O)c1cccc(C(F)(F)F)c1', 'Clc1cccc(S(=O)(=S)Nc2ccccc2C(F)(F)F)c1', 'Cc1ncsc1[C@@H](O)N1CCN(Cc2cccc(F)c2)CC1', 'N=Cc1ncnc2cc(-c3cc(C)cc(OC)c3)nn12', 'Oc1ccc([N+](=O)c2cc(Cl)ccc2O)c2ncnn12', 'Oc1cc(F)nc(-n2cnnn2)c1Cl', 'Oc1cc(-n2nncc2F)nc(-c2ccccc2Cl)n1', 'O[C@@H]1C[C@H]1C(=O)Nc1ccc(Cl)s1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccn1Cc1cccc(Cl)c1', 'Clc1ccc(Br)cc1[C@@H](O)[C@@H]1CCOC2(CCC2)C1', 'Clc1ccc(Br)c(N[C@H]2CCOC3(CC[NH2+]CC3)C2)c1', 'Clc1ccccc1O/C=C(/C#N)C(=O)N1CCCC1', 'C=CC[NH+](CC(=O)[O-])Cc1ccc(OC)c(OC)c1', 'Oc1cccc2ncc(C[NH2+][C@@H]3CCSc4ccccc43)n12', 'Oc1cc(F)cc(/C=C/C[NH2+]Cc2ccc(F)cc2)c1', 'Clc1cccc2nn(S[C@@H](CO)c3ccccc3)nc12', 'Clc1ccc(-c2nonc2N2CCc3sccc3C2)cc1', 'Clc1ccsc1/N=c1\\sccn1Cc1cccc([N+](=O)[O-])c1', 'Clc1ccccc1-c1csc([C@@H]2C(=O)NC(=O)c3ccc(Cl)cc32)n1', 'Clc1cc(NSc2cccc(Cl)c2Cl)ccc1OC(F)(F)F', 'N#CCSc1cccc(OCc2ccccc2)c1', 'O=C(c1cccs1)c1csc(NCc2cccc(Cl)c2)n1', 'Oc1cc(-n2cn/nc3c4c(cccc24)OC[C@@H]3CO)cc1=O', 'Clc1cccc2nsnc12', 'O=C(CNS(=S)(=O)c1ccc(Br)cc1)Nc1cccc([N+](=O)[O-])c1', 'Nc1c(Oc2cccc(C(=O)N3CCOCC3)c2)cnn1N', 'O=C(c1cc(Cl)cc(OC(F)F)c1)N1CCOCC1', 'O=[N+]([O-])c1c(F)cccc1Cl', 'Clc1ccccc1N1Nc2ccccc2NC1=O', 'ClCc1nnsc1Cl', 'ClCc1nnsc1Cl', 'Oc1cc(-n2nnc(C3CC3)n2)c(Cl)cc1Cl', 'Clc1ccc(Br)cc1[C@@H]1C[NH+](C2CC2)CN1Cc1cccc(F)c1', 'Clc1ccc([C@@H](N[C@@H](C#N)c2ccccc2)C(F)(F)F)cc1', 'Clc1ccsc1-c1cccc(C(F)(F)F)c1', 'ClC(c1cc(Cl)ccc1Cl)N1CCc2ccc(F)cc2C1', 'NS(=O)(=O)Cc1noc(I)c1[N+](=O)[O-]', 'O=C(/C=C\\NCc1csc([N+](=O)[O-])c1)c1ccccc1', 'ClCc1csc2nc(-c3ccccc3)nn12', 'Oc1cccc(N2C(=O)[C@H]3CN(c4ccc(Br)cc4Cl)CN3C2)c1', 'Clc1ccc(Cc2noc(-c3ccc(Cl)o3)n2)c(F)c1', 'ClCc1ccn(C(=O)c2ccc(F)cc2Cl)c1', 'Clc1ccc(CNc2ccc3c(c2)OCO3)cc1', 'Clc1ccccc1[C@H]1[C@H]2NC(=S)N=C2S1', 'Clc1ccccc1OCC(F)(F)F', 'O=C(COc1ccc(Cl)cc1)Nc1ccc(F)c(C(F)(F)F)c1', 'Clc1ccccc1N1N=C(Sc2ccccc2Cl)SC1=S', 'O=[N+]([O-])c1ccc([C@H]2CSC[C@@H](Cc3cccs3)O2)c(Cl)c1', 'ClCc1nnc(CNS(=O)(=O)[C@H]2CCCc3ccccc32)s1', 'Oc1ccc([N+](=O)c2ccccc2)c(Oc2cccc(F)c2)c1', 'Clc1cccc(Br)c1C(F)(F)F', 'Clc1ccccc1C(=O)Nc1nnc(-c2ccc(Cl)c(Cl)c2)s1', 'NS(=O)(=O)/C(=C\\Cc1cccc(C(F)(F)F)c1)c1cccc2ccccc12', 'O=C(CNc1cc(F)c(F)cc1[N+](=O)[O-])Nc1ccccc1F', 'ClCc1csc(-c2ncc(Br)s2)n1', 'Clc1nncn1c1c(-c2ccc([N+](=O)[O-])cc2)cc(Br)cc1O', 'O=C(CSCNC(=O)c1ccc([N+](=O)[O-])cn1)Nc1cnc(-c2cccnc2)o1', 'O=C(CNC(=O)c1ccc([N+](=O)[O-])cc1)NNC(=O)c1ccccc1[N+](=O)[O-]', 'O=C(c1cc(F)cc(F)c1)N1CCC(n2cncn2)CC1', 'Clc1cc(N2CCN(S(=O)(=O)c3ccc(Cl)cc3Cl)CC2)ccc1[O-]', 'Oc1ccccc1-c1c([O-])cc([N+](=O)[O-])c2noc(=O)n12', 'O=C(c1ccsc1)c1nnc(-c2ccccc2)s1', 'ClCc1cc(CO)oc1[O-]', 'Clc1cccc2[nH]nc(Sc3cccnc3)c12', 'O[C@@H](c1cc([N+](=O)[O-])ccc1F)[C@H]1C[C@@H]1c1cc(Cl)ccc1Br', 'Clc1ccc(Br)c([C@H]2c3ccccc3C=CN2C(=O)c2ccc(O)cc2)c1', 'Clc1ccc(Br)cc1SC/C=C\\c1cnc(Oc2ccccc2)c([O-])c1C(=O)[O-]', 'O=C(Cc1nc(Cl)ccc1Cl)c1cccc(Cl)c1', 'Clc1ccc(-c2ccccc2)cc1O', 'Oc1ccc([N+](=O)Cc2ccccc2)cc1Cl', 'Oc1cc(O)ccc1O[C@H]1CCCc2ccccc12', 'O=[N+]([O-])c1c([N+](=O)[O-])c(-c2ccccc2)nc2sccc12', 'N#C[C@@H]1[C@H](c2cccc(Br)c2)[C@@H](c2ccccc2)OC1=O', 'Clc1ccc(-c2cc(Br)oc2C2CC2)o1', 'O=C(c1ccsc1)[C@@H]1SC(N2CC2)c2ccccc2S1(=O)=O', 'Clc1cc(Cl)c2scnc2n1', 'ClCc1nnc(-c2cccs2)o1', 'Oc1ccc(F)c(F)c1SCc1nc2ccccc2c(=O)[nH]1', 'N#C[C@@]1(O)CCc2c(s1)C[NH+](Cc1csc(-c3cccs3)c1O)CC2', 'Clc1ccccc1NC(=S)NCc1cccc(C(F)(F)F)c1', 'O=C(c1cnn(-c2ccccc2Cl)c1)N1CCC(C(F)(F)F)CC1', 'Clc1ccccc1Oc1cc(Br)cc2ccccc12', 'Oc1cccc(CCl)c1', 'O=[N+]([O-])c1cc(Cl)nc(Nc2ccccc2Cl)c1', 'NS(=O)(=O)CCNC(=O)c1cccc(NC(=O)c2ccco2)c1', 'Clc1ccc(COc2ccc(F)cc2)cc1F', 'Oc1cc(O)c([N+](=O)[O-])cc1OC[C@H]1CCCO1', 'O=C(CSCc1cc(=O)[nH]c(-c2ccccc2)n1)NC[C@H]1CCCO1', 'NS(=O)(=O)C[C@H](c1cccs1)n1nnc2cc(Cl)ccc21', 'Oc1cccc([C@H](O)Cc2nc3ccccc3s2)c1', 'Clc1ccccc1Nc1ccc(F)cc1F', 'Clc1cccc([S@@]C2=NC(=O)[C@@H]3CCC(=O)N32)c1', 'Clc1nnnn1-c1cccc(Cl)c1', 'Clc1ccc(C2CC2)cc1F', 'Clc1ccc(C2CC2)cc1F', 'Cc1ncc2cc(C(=O)N[C@H](C#N)C(N)=O)cc(=O)cc2n1', 'O=C([O-])/C=C/Sc1cccc(F)c1[N+](=O)[O-]', 'ClC(/C=C(/C#N)c1ccccc1)Nc1ccccc1', 'Clc1ccsc1Nc1nc2ccc(F)cc2s1', 'ClCc1n[nH]c([C@@H]2c3ccccc3C[C@]2([NH3+])C(=O)[O-])c1', 'O=C([O-])CS/N=C1\\C(=O)Nc2c1cnc1ccccc21', 'O=C([C@H](Cl)Br)S(=O)c1ccc2ccccc2c1', 'ClC(=S)Oc1ccc(Cl)cc1Br', 'ClCc1cc(F)ccc1C1CC1', 'Clc1ccc(Br)cc1[C@@H]1C[C@H]1c1ccccc1F', 'Clc1ccc(Br)c(C[NH+]2CC[C@@H]3CCCC[C@@H]32)c1', 'Oc1cccc(Nc2[nH+]c3ccc(F)cc3s2)c1', 'Clc1ccccc1C(=O)N1CCCc2ccc(F)cc21', 'Oc1cc(Br)c2ccccc2c1C(=O)N1CCCCCC1', 'Clc1cccc2ncn(CCCSc3ccccn3)c12', 'Clc1ccc(-c2cccnc2S(=O)(=O)N2CCCCC2)cc1', 'Clc1ccccc1Oc1cc(F)ccc1F', 'Clc1ccccc1Oc1cc(Cl)ccc1F', 'Clc1ccccc1Oc1cc(Cl)ccc1Cl', 'Clc1ccccc1Oc1cc(Cl)cc(Cl)c1', 'O=[N+]([O-])c1cc([C@H]2[C@H]3CCCC(=O)[C@H]3[C@H]3CCCC[C@H]3O2)ccc1C(F)(F)F', 'Clc1cc(Cl)ccc1CSc1ncccn1', 'O[C@@H]1c2cc(OC(F)(F)F)[nH]c2N[C@H]1C(=O)[O-]', 'Clc1ccccc1[C@H]1[C@@H]([O-])CC(O)=N[C@@H]1N1CCOCC1', 'Clc1nnn(-c2ncc3nc[nH]c3c2)c1Cl', 'Clc1nnc(NC(=O)c2ccc3c(c2)OCO3)[nH]1', 'Clc1cccc2ccc(/C=C/O)cc12', 'Clc1ccc(Br)cc1-c1cnnn1C(F)(F)F', 'Clc1nnnc(-n2nc(-c3ccc(F)cc3)cc2Br)n1', 'O=C(/C=C1/N=NC(=O)N(C2CCC2)C1=O)c1ccc2c(c1)COC2', 'Clc1ccsc1N[C@@H]1CCCN(c2ccccc2F)C1', 'Clc1ccc(Br)cc1-n1nnc(C(F)(F)F)n1', 'O=C([O-])/C(=N/c1c(Cl)cccc1[N+](=O)[O-])N1CCCC1', 'CCCC[C@H](CC)c1ccc([N+](=O)[O-])cc1', 'O=[N+]([O-])c1ccn(Cc2cc(F)cc3c2OCOC3)c1', 'ClCc1csc(Sc2cc(F)ccc2Br)n1', 'O=C(CSCNc1ccccc1C(F)(F)F)NCc1ccccc1', 'Oc1cccc2ncn(/N=C/c3c(Cl)cccc3F)c12', 'Clc1cc(/N=C/Cc2ccccc2Cl)ccc1O', 'O=[N+]([O-])c1cc(C2/N=CC=C[C@@H]2Oc2ccccc2)ccc1Br', 'CNS(=O)(=O)c1cc(Cl)c(C(=O)N2CCC[C@@H]2c2cccnc2)cc1Cl', 'Oc1cc(Cn2ccnn2)c2cn[nH]c2c1O', 'O=C([O-])/C(=N/n1cccn1)Nc1ccc(Br)cc1C', 'Oc1ccc([N+](=O)[O-])c(N/N=C\\c2ccccc2F)c1', 'COc1nc2ncc(C)cc2s1', 'O=C([O-])/C(CCC(=O)Nc1ccc2ccccc2c1)c1ccc(C#N)cc1', 'Clc1cccc2[nH]nnc1c1nc2ccccc1', 'CNC(=O)Cc1cc(=O)oc2ccc(Cl)cc12', 'Clc1ccc(Br)cc1-c1cc(-c2cccnc2)ncn1', 'CN[C@H](C(=O)n1ccc(=O)[nH]c1=O)c1ccc(C)c(C)c1', 'Clc1ccc(Br)c(F)c1F', 'Clc1ccc(Br)c(F)c1F', 'Clc1ccc(Br)c(Cl)c1Cl', 'Clc1ccc(Br)c(Cl)c1Cl', 'Clc1ccc(Br)c(Cl)c1Cl', 'O=[N+]([O-])c1cccc2nc(-c3cc(Cl)ccc3[N+](=O)[O-])[nH]c12', 'Nc1c(-c2cccc(Cl)c2)ncnc1Nc1cc(F)ccc1F', 'Nc1ncc2c(c1O)C[NH+](Cc1cccc(C(F)(F)F)c1)CC2', 'O=[N+]([O-])c1ccc([C@H]2C[C@H]2CN2CCOCC2)cc1', 'Clc1nnc(Br)s1', 'Oc1ccc(Oc2cnc3c(c2)C[C@H](c2ccc(Br)s3)cc2)C1', 'Clc1cc(CC(=O)N2CCC[C@@H]2c2ccco2)ccc1F', 'Clc1ccc(Br)cc1N1C[C@H](c2ccccc2)CC1', 'Clc1ccc(Br)c(CN2CCc3ccccc3C2)c1', 'COC(CC)(NC(=O)c1ccc(C)c(C)c1)C(=O)OC', 'O=C(/N=C/Nc1ccc(NSc2ncccc2F)cc1OC(F)F)Nc1ccc(C(N)=O)cc1', 'Clc1cc(Br)cc2ncnc(-c3cccs3)c1n2-c1ccccc1', 'O=[N+]([O-])c1cccc2[nH]cnc1c1cc(F)c(Cl)cc12', 'Oc1cccc(C(=O)C2=CC(CCOc3ccc4c(c2)OCCO4)C3)c1', 'O=C(c1cc(Br)c2c(c1)OCCO2)N1CCC[C@@H]1c1cccc(F)c1', 'Clc1cccc2cc(F)c(SCc3cc(C(F)(F)F)ncn3)cc12', 'Clc1cc(F)ccc1-n1ncc(C(F)(F)F)n1', 'O=C(/C=C\\N=C1\\Nc2ccccc2C1=O)c1ccccc1Cl', 'NS(=O)(=O)C[C@H](O)c1ccc(Cl)c(Cl)c1', 'Clc1nnnn1c1nc(-c2ccco2)cs1', 'ClCc1nnsc1Br', 'Oc1cc(Br)ccc1-c1cccc2ccccc12', 'N#Cc1ccc(-c2n[nH]cc2-c2cccc(C(F)(F)F)c2)cc1', 'Clc1cccc2ncn(C[C@H](O)c3ccco3)c12', 'O=C(Cc1cn2ccsc2n1)NNC(=O)c1ccc(Cl)s1', 'O=C(CNc1nn(-c2cccc(C(F)(F)F)c2)c2c1NCCC2)Nc1ccc(F)c(F)c1', 'O=[N+]([O-])c1c(-c2cc(F)ccc2O)n[nH]c1[O-]', 'O=C([C@H](O)c1ccc(Cl)c(Cl)c1)N1CCN(C(=O)c2cccc(F)c2)CC1', 'Oc1cc(-n2cc(Br)cn2)ccc1C(F)(F)F', 'NS(=O)(=O)Cc1cncc([N+](=O)[O-])c1', 'N=C1\\[C@@H](c2cccc(F)c2)NC(=S)NC1=O', 'Clc1cccc2cc(O[C@H](CSC(F)(F)F)C(F)F)ncc12', 'Oc1cc(Br)cc([NH2+][C@H]2CCc3ccc([N+](=O)[O-])cc32)c1', 'O=[N+]([O-])c1ccc2ncn([C@@H](c3ncccc3Cl)C(F)(F)F)c2c1', 'O[C@@H]1C(=O)N(Cc2cnc3ccccc3c2[O-])C=C[C@@H]1O', 'O=C(c1cc(Cl)c2cccnc2n1)N1CC[C@@H](c2ccccn2)C1', 'Oc1cc(F)nc([S-])c1SCc1ccccc1Cl', 'C=CCOc1cc([N+](=O)[O-])ccc1OCc1ccc(OC)cc1', 'Clc1ccccc1NC(=S)N1/C(=N/N=C\\c2ccc(C(F)(F)F)s2)SC1=S', 'Clc1ccc(CCC(F)(F)F)cc1', 'Clc1nnn(C(F)F)c1C(=O)Nc1ccc2c(c1)N(C)c1ccc(Cl)cc1C(=O)N2', 'NS(=O)(=O)/N=c1\\oc2c(cnc3ccccc13)C(=O)CCC2', 'O=[N+]([O-])c1c(-c2cccs2)nnn1-c1ccccc1', 'O=C(/C=C/O[C@@H]1COc2ccccc2O1)C1=NN(c2ccccc2)SC1=O', 'OC[C@H](Cl)c1ccc2ccccc2c1', 'O=C(CN(c1cnccc1)S(=O)(=O)[O-])C1CCCC1', 'O=C(/C=C/c1ccc2c(c1)OCO2)NCc1cccs1', 'O=C(/C=C1CS(=O)(=O)C[C@H]1O)c1ccccc1Br', 'Nc1cnn(C2CCN(c3ccc(C#N)cc3)CC2)n1', 'CN[C@H](c1[nH+]cc(C)s1)c1ccc(Br)s1', 'Oc1cc(-n2nncc2[C@@H]2CSCc3ccccc32)cs1', 'Oc1cccc(O[C@H]2CCCCO2)c1', 'O=C(CC(Nc1ccccc1)N1CCOCC1)N[C@@H](c1ccc([N+](=O)[O-])cc1)C(F)(F)F', 'Clc1cccc([S@@](=O)/C=C/c2ccc([N+](=O)[O-])cc2)c1', 'Oc1cccc(SCS(=O)(=O)c2ccsc2)c1', 'O=C(/C(c1cccs1)Sc1ccccc1)c1ccno1', 'Oc1ccc([N+](=O)[O-])c(N[C@H]2CCc3cc(F)ccc32)c1', 'Clc1ccccc1-c1cscc1-n1nccc1SCc1ccccc1F', 'O=[N+]([O-])c1c(O)sc2c1CCN(Cc1cc[nH]n1)C2', 'Clc1ccc(CN2N=C3c4ccccc4O[C@@H]3C2)s1', 'Clc1ccccc1Oc1cc(N2Sc3ccccc3CC2)n[nH]1', 'O=C(c1ccsc1)c1cnc(-c2cccc(OC(F)F)c2)nn1', 'Oc1cc(F)c(-c2cccnc2)nc1CO', 'Clc1ccc(OC[C@H]2CO2)cc1F', 'O=C(/N=c1\\occc(-c2ccccc2)n1)c1cccc(F)c1', 'Clc1ccc(Br)cc1N1CCCCCC1', 'Clc1ccc(Br)c(N2CCCCCC2)c1', 'O=[N+]([O-])c1ccc(C[NH2+]Cc2ccc(O[C@@H]3CCOCC3)o2)cc1', 'Cc1sc(C(=O)N/N=C/c2ccc(C(C)C)cc2)c(C#N)c1C', 'O=C(/C(c1cccs1)=C1/N=C(c2ccc(Br)cc2)SC1=S)c1ccccc1', 'Clc1ccccc1Nc1cc([C@H]2CS(=O)(=O)CCN2)ncn1', 'NS(=O)(=O)Cc1noc(-c2ccc(F)cc2)n1', 'Clc1ccc(C[NH2+]c2cccc(C(=O)[O-])c2)cc1', 'ClC[C@@H]1CCO[C@]1c1c(F)cccc1F', 'Nc1cn[nH]c1CSc1n[nH]c(=O)[nH]1', 'Clc1ccc(Br)cc1-c1ccc(CO)o1', 'NS(=O)(=O)/N=C/C(=O)Nc1ccc(Cl)cc1', 'O=[N+]([O-])c1c(Br)c(-c2cccc(O)c2)nc2[nH]ccc12', 'Nc1[nH]nc2c1SC/C2=C(\\N)Nc1ccccc1', 'O=[N+]([O-])c1ccnc2c1nnn2-c1cccc(C(F)(F)F)c1', 'Oc1ccc(F)c(-c2noc(-c3ccsc3)n2)c1', 'OCS(=O)(=O)c1c(Cl)cccc1Cl', 'Clc1ccc(Br)cc1/N=N\\OCc1ccc(Cl)cc1Cl', 'Clc1ccc(C2=Cc3ccc([N+](=O)[O-])cc3O2)cn1', 'Clc1ccc(-c2cc(-c3cc[nH]n3)ncn2)cc1', 'Oc1ccc(Oc2cc([N+](=O)[O-])ccc2N2CCOCC2)cc1', 'Clc1ccc(C2CCN(C(=O)C(F)(F)F)CC2)cc1', 'Oc1cccc(S[C@@H]2NC(=O)COc3ccc(Cl)cc32)c1', 'ClCc1ccc2nc(-c3ccncc3)n[nH+]2c1', 'O=C([O-])c1nn(C(=O)c2ccc(Br)cc2)c2ncc(C(F)(F)F)cc12', 'Clc1cc(/C=N\\NC2=N[C@H]3OCO[C@@H]32)ccc1Br', 'OCc1nnsc1Br', 'O=C(c1ccc(Cl)cc1)N1CCOCC1', 'NS(=O)(=O)c1ccc(CNC(=O)c2cccnc2)cc1', 'O=[N+]([O-])c1ccncc1[C@@H]1CSC(F)(F)C1=O', 'O=C(/N=C/c1ccc([N+](=O)[O-])cc1)C(F)(F)c1ccccc1', 'O=C([C@H](Cl)c1ccccc1)c1ccc([N+](=O)[O-])cc1F', 'O=[N+]([O-])c1c(Br)sc2[nH]cnc12', 'Oc1ccc([N+](=O)[O-])c(C(=O)c2ccc(NCCO)cc2)c1', 'Clc1ccccc1C(=O)O[C@H]1C=NN=C[C@@H]1c1ccccc1', 'Clc1cc(Br)c2ccccc2c1Cl', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1[O-]', 'Nc1n[nH]c(/N=C2\\C(=O)NC(=O)C([O-])=CC2=O)n1', 'O=C(CSCc1ccc(O)cc1)Nc1cccc(OC(F)(F)F)c1', 'Oc1cc(-n2nnc(Cc3ccc(Br)s3)n2)ccc1[O-]', 'Clc1cc(F)c(I)s1', 'O=[N+]([O-])c1ccc(/C=C\\c2cc(Br)ccc2O)cc1O', 'Oc1ccc(O[C@@H]2C(=O)Nc3ccccc32)c(Cl)c1Cl', 'Clc1cc(Br)c2nc(-c3cc(Cl)cs3)oc2c1', 'Clc1ccc(C(=O)OC[C@@H](O)c2cccc(Cl)c2)cc1', 'Oc1ccc([C@H]2C(C#N)=C(Nc3ccccc3)NC(=O)N(c3ccc([N+](=O)[O-])cc3)N2)cc1', 'CN(C)C(=O)C(=O)Nc1ccc(-c2nc3ccccc3s2)c(C(F)(F)F)c1', 'Clc1ccc(CN2CCc3ccccc3C2)c(F)c1', 'Clc1ccccc1NC(=S)Nc1nncs1', 'Clc1ccsc1-c1cc(COc2cccc(Cl)c2Cl)on1', 'OCc1nn(-c2ccccc2)c(=S)n1C1CC1', 'N#Cc1c(F)cc(CNC(=O)NC[C@H]2CC=CCC2)cc1C(F)(F)F', 'Oc1ccc([N+](=O)c2cc(Br)ccc2C(F)(F)C[NH+]2CCCC2)cc1', 'O=C(C1=Cc2cc(F)ccc2OC1)N1CCCc2ccsc21', 'O=[N+]([O-])c1cc(N2CCOCC2)ccc1N1CCOCC1', 'OCc1nnsc1C(=O)c1ccc(Br)s1', 'Clc1cccc(C(F)(F)F)c1', 'Clc1cc(COC(=O)c2ccco2)ccc1[N+](=O)[O-]', 'Clc1nncc(OCc2cccc([N+](=O)[O-])c2)n1', 'Clc1cc(Nc2c(Cl)cccc2Cl)ncn1', 'Clc1cc(/N=C/c2cccc([N+](=O)[O-])c2)sc1O', 'COc1ncc(-c2ncc[nH]2)cc1C#N', 'N[C@@H]1NC(=O)/C1=C/c1ccc(-c2ccc(F)cc2Cl)o1', 'Clc1nnnn1-c1cc(S(=O)(=O)N2CCCC2)ccc1Cl', 'Clc1nnnn1-c1cc(S(=O)(=O)N2CCCC2)ccc1Cl', 'O=[N+]([O-])c1cc(Nc2ncccc2F)ccn1', 'O=[N+]([O-])c1c(F)cc2n1CCC[C@@H]2COC1CC1', 'O=[N+]([O-])c1ccc(Oc2cc(C3CCCCC3)co2)cc1', 'Oc1cccc2cc(OCC(=O)Nc3cc(C(F)(F)F)ccc3O)ccc12', 'Clc1cccc2cc(CNc3nccs3)ccc12', 'Oc1ccc(F)cc1OS(=O)(=O)c1ccccc1O', 'Oc1ccc(F)c(Cl)c1Sc1ccc(Br)cc1O', 'Clc1ccccc1-c1nc(CO)[nH]c2cc(-c3ccco3)[nH]c2c1N', 'Clc1cc(/C=C2\\C(=O)NC(=S)N(c3ccccc3)C2=O)ccc1Cl', 'Oc1cccc(SCc2nc(-c3ccccc3)no2)c1', 'O=C(COc1cc(C(F)(F)F)sc1-n1ccnc1)c1ccc(C#N)cc1', 'Clc1cc(NS(=O)(=O)c2ccccc2)c2ccccc2n1', 'O=[N+]([O-])c1c(F)ccc(Nc2ccccc2)c1', 'Clc1nnnc(Sc2ccnc(N3CCOCC3)c2)n1', 'Clc1nnnc(Cc2ccc(Br)s2)c1N', 'Clc1ccc([C@@H]([O-])Nc2ccc(Br)cc2Cl)cc1', 'Clc1ccc(Br)cc1O[C@H](N)c1ccc(F)cc1F', 'Clc1cc(Cl)c(C(=O)Br)cc1Oc1ccc(F)cc1', 'Clc1cc(Cl)nn1/N=C(/[O-])c1ccc2c(c1)OCO2', 'O=C(COc1cc(Cl)c(Cl)cc1Cl)c1ccc(C(F)(F)F)cc1', 'Clc1cc(Cl)c(S(=O)(=O)C(F)F)c(Cl)c1Cl', 'Clc1ccc(Br)cc1/C=N\\[C@H]1CCC[C@H](O)C1', 'Clc1nncc(Sc2nnc3ccc(F)cc3c2Cl)n1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(-c2ccc(F)cc2)c1N', 'Clc1ccccc1NS(=O)(=O)c1c[nH]nc1C(F)(F)F', 'Clc1ccccc1O/N=[N+]1/N=C/SCCS1', 'O=[N+]([O-])c1c(Br)cccc1C(F)(F)F', 'CN[C@H](c1cccs1)c1cccc(-n2cncn2)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccn1[C@H]1CCOc2ccc(F)cc21', 'O=[N+]([O-])c1c([N+](=O)[O-])cnnc1N', 'Nc1cnnn1/C(Cl)=N/C=C/C(=O)[O-]', 'NS(=O)(=O)Nc1ccc([N+](=O)[O-])cc1Cl', 'Oc1ccc([N+](=O)[O-])c(N2CCOCC2)c1F', 'Clc1cc(/N=C\\c2c[nH]c[nH+]2)cs1', 'Cc1sc(N)c([C@@H]2CC[NH+](Cc3cccc(F)c3)C2)n1', 'Clc1cccc(Nc2nccs2)c1Cl', 'NS(=O)(=O)/C=C(\\Cc1ccccc1)I', 'ClCc1ccnn1-c1ccc([N+](=O)[O-])cc1', 'Clc1ccccc1Cc1c(F)cc(Cl)cc1F', 'Clc1cccc(CN2CC[NH+](CC[C@H]3CC[NH+](Cc4ccc([N+](=O)[O-])cc4)CC3)C2)c1', 'Clc1cc(/N=C/c2sccc2C(F)(F)F)ccc1Cl', 'O=C([C@H](NS(=O)(=O)c1ccc([N+](=O)[O-])cc1)c1cccs1)Nc1ccccc1C(F)(F)F', 'Cc1sn(-c2ccc(F)cc2)c(=S)c1/C=N/NC(N)=O', 'N#C/C(=C\\c1cnc(-c2ccco2)cc1O)[C@@H]1CCCO1', 'O=C(c1cccc(Br)c1)Oc1ccc(Cl)cc1', 'Oc1cccc([C@H]2O[C@H](Br)[C@H](F)[C@H](O)C2)c1', 'Clc1nncnc1[N-]C(=O)c1ccc(C#N)cc1', 'O=[N+]([O-])c1c(Br)cccc1-n1cc([O-])cn1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1Sc1ccccc1F', 'Oc1cc(Br)c2sccc2c1OCc1ccc(O)cc1', 'O=C(COc1ccc([N+](=O)[O-])cc1)N[C@@H]1CCCN(c2ccc(F)cc2)C1', 'Clc1cccc(S(=O)(=O)N2CCC(C(=O)OCc3ccsc3)CC2)c1', 'ClCc1csc2nc(SCc3ccc(Cl)cc3)nn12', 'ClCc1csc([N-]S[C@H]2C=CCC2)n1', 'Clc1cccc2cc([C@H](O)CSc3nc4ccccc4s3)oc12', 'O=[N+]([O-])c1ccc2[nH]cc(-c3ccn[nH]3)c2c1', 'Clc1ccccc1OCc1cc(=O)n2nc(Br)sc2n1', 'N=Cc1nonc1S(=O)(=O)Cc1cccc([N+](=O)[O-])c1', 'O=C(/C=C1CCSc2ccccc21)c1cc(F)ccc1F', 'Clc1nnnc(-c2ccc(Cl)cc2)c1-c1cccc(C(F)(F)F)c1', 'Clc1ccc(-c2noc(-c3cccc(C(F)(F)F)c3)n2)c(Cl)c1', 'O=[N+]([O-])c1ccncc1O/N=C1\\C(=O)Nc2ccccc21', 'Oc1ccc([C@H]2CCCC(C#N)(C3CCOCC3)c3ccccc3N2)cc1', 'O[C@@H](c1csnc1)[C@H]1CC(=O)N(c2cc(Cl)ccc2C#N)C1', 'Clc1cccc2[nH]nnc1c1nnc(NCc3ccccc3)n12', 'Clc1cc(Br)cnc1O/N=C1\\C(=O)N[C@@H](c2cc(Cl)c(O)cc2)[C@H]1O', 'OCc1nn(Sc2ccc(Cl)cc2Cl)c2ccccc12', 'O=C(Cc1c[nH]c(C(=O)[O-])c1)Nc1cccc([N+](=O)[O-])c1', 'Nc1nc2sc3ccccc3n2c1NCc1cccnc1', 'Clc1ccccc1NC(=O)[C@H]1COc2ccccc2O1', 'ClCc1nnc(-c2ccccc2)n1-c1cccc(Cl)c1', 'O=C(c1cc(Br)cs1)N1CCN(c2ccccc2C(F)(F)F)CC1', 'NS(=O)(=O)/C=C/C(=O)c1ccccc1', 'O=C(C[NH+]1CCc2sccc2C1)Nc1cccc(F)c1', 'Oc1ccc(F)c(O[C@@H]2CCOC3(CCCC3)C2)c1', 'Clc1ccsc1/N=C1/S[C@H](CC(=O)NC2=NN=CC3)CCCC3=C2C=C1', 'Clc1ccccc1[C@H]1NC(=O)NC(=S)N1', 'N#Cc1c(Br)cccc1N[C@H]1CCSC1=O', 'Clc1ccccc1Nc1cc(Br)cc(F)c1F', 'NS(=O)(=O)C[C@H](NC1=NS(=O)(=O)c2ccccc21)c1ccccc1', 'Clc1cc(F)c(Cl)cc1N[C@@H]1CCOc2ccccc21', 'Clc1cc(NSc2ccc(Cl)s2)ccc1C(F)(F)F', 'Clc1ccsc1C[N-]c1ccc2[nH]ccc2c1', 'O=C([O-])c1cnc(=S)n(-c2nccc(Oc3cccnc3)n2)c1', 'Oc1cccc([C@H]2Oc3c(F)ccc(F)c3[C@H]3[NH2+]CCC[C@H]32)c1', 'O=C(c1cccs1)C1CCN(c2nc3ccccc3oc2=O)CC1', 'Clc1cccc2cc(O)ccc12', 'Oc1cccc(-c2csc(COc3ccc(F)cc3)n2)c1', 'NS(=O)(=O)Cc1cn2ccsc2n1', 'O[C@@H]1[C@@H](NS(=O)(=O)c2ccc3c(c2)C(=O)c2ccccc2S3)N=C2CCCC[C@@H]21', 'NS(=O)(=O)/C=C/c1cccnc1', 'O=C(c1cccs1)C1=CC(c2cccc(Cl)c2)SC(=S)N1c1ccccc1', 'Clc1cccc(Nc2cc(F)cc(Cl)c2F)c1', 'O=[N+]([O-])c1c(F)cc2c(C(F)(F)F)ncnc2c1', 'O=[N+]([O-])c1ccc(O[C@H]2c3ccccc3C=C[C@@H]2O)cc1', 'Oc1cccc(SCCc2nnc(-c3ccccc3[N+](=O)[O-])o2)c1', 'Clc1ccc(-c2nnc(-c3ccc(=O)ccc3CO)s2)c(O)c1', 'Clc1ccsc1Sc1ccc(Cl)c(C(=O)[O-])c1', 'O[C@@H](C[C@H]1CCCN(S(=O)(=O)c2ccc(C(F)(F)F)cc2)C1)c1cccnc1', 'Oc1ccc(/C=N/Nc2ccc(Cl)cc2Cl)c(Br)c1', 'Oc1cc(-n2nnc(-n3cccn3)n2)ccc1F', 'Clc1nncc(S/[N+]=C/C=CO)c1I', 'Clc1ccccc1Oc1cc(Cl)c(Cl)cc1Cl', 'Clc1ccccc1Oc1cc(Cl)c(Cl)cc1Cl', 'Clc1ccccc1Oc1cc(Cl)c(Br)cc1F', 'Clc1ccccc1Oc1cc(Br)c(F)cc1Cl', 'Oc1cc(O[C@@H]2CCO[C@@]3(CCOC3)C2)c(Cl)cc1Cl', 'Clc1ccc(Br)cc1O[C@@H]1CCOC2(CCC2)O1', 'C=CCNC(=O)c1ccc(C)c(OC)c1', 'NS(=O)(=O)Cc1nnn(-c2ccccc2)c1N', 'O=[N+]([O-])c1c(Br)scc1C(F)(F)F', 'Clc1ccc(Br)cc1N1/C=C/Nc2ccccc21', 'Clc1cccc(Br)c1S(=O)(=O)C1=c2ccccc2=[NH+]C1', 'Clc1ccc(-c2ccc(Cl)c(F)c2)cc1Br', 'Clc1ccc(-c2cc(F)cc(Br)c2)cc1F', 'O=C(CNc1nnc(SCc2ccccc2)o1)Nc1cccc(Cl)c1', 'O=C(Cc1cnc[nH]1)NNC(=O)c1cccs1', 'O=C(Cc1c(F)cccc1F)c1ccc2c(c1)OCO2', 'Clc1cc(Br)ccc1NC1(F)CCCCC1', 'Clc1cc(F)c(Cl)cc1C(=O)NNc1ccccc1', 'Clc1nnn(-c2cn(Cc3cccc(C(F)(F)F)c3)cn2)c1C#N', 'Cc1nc(C(C)C)c2nc(Nc3ccc(Cl)cc3)sc2n1', 'Clc1cccc([S@@]C2=NCCc3ccccc32)c1', 'Clc1ccc(-c2csc3[nH]cc(Br)c23)c(Br)c1', 'Oc1cc(Br)cc(OCCO)c1', 'O=[N+]([O-])c1cc(Cl)c2nc(N)sc2c1', 'Oc1cccc(O[C@@H]2CCOc3ccc(F)cc32)c1', 'Clc1cc(Cl)nnc1CSc1ncc[nH]1', 'Clc1cc(Br)cnc1SCc1ncn[nH]1', 'O=[N+]([O-])c1c(Cl)nnc2sc(-c3ccco3)nc12', 'Clc1cc(F)cc(C(=O)N2Cc3ccccc3[C@@H](c3ccccc3)C2)c1', 'Clc1nnnn1/N=C/c1cc(Cl)ccc1O', 'Clc1nnn(-c2cncc(C(=O)[O-])c2)c1Cl', 'Clc1cc(CCl)c(F)cc1F', 'Oc1ccc(F)cc1Sc1nc2ccc(C(F)(F)F)cc2s1', 'O=C(c1ccsc1)[C@@H]1CC(=O)N(Cc2cccc(F)c2)C1', 'Clc1cccc(SCc2nnnn2Cc2ccccc2)c1', 'Clc1cc(F)cc(N[C@@H]2CCCC[C@@H]2O)c1', 'Clc1cc(Cl)nc([N-]S(=S)(=O)c2ccc(F)cc2)n1', 'Clc1cccc(N2CC[C@@H](C(=O)[O-])C2)c1', 'Oc1ccc(O[C@@H]2CCC[NH2+]C2)cc1Cl', 'Oc1ccc(Cl)c(C(=O)N2CCCCCC2)c1', 'Clc1cccc(C(=O)NC2CC2)c1', 'Clc1ccc(Br)cc1OC1CC[NH2+]CC1', 'C=CCC[C@@H](C)OC(=O)c1ccc(-c2nc(C)c3ccccc3n2)s1', 'O[C@@H](c1csnc1)c1cc(F)c(F)cc1Cl', 'O[C@@H]([C@@H]1COc2ccccc21)c1cc(F)ccc1F', 'O=C(/N=C\\c1ccccc1[N+](=O)[O-])Nc1ccc([N+](=O)[O-])cc1', 'Oc1ccc([N+](=O)c2cc(CCO)c(C(=O)[O-])cc2Br)cc1', 'Clc1ccccc1Oc1cc(Br)ccc1O', 'Clc1ccccc1Oc1cc(Br)ccc1O', 'Clc1cc(/N=C\\c2ccccc2Br)ccc1O', 'Clc1nnnn1-c1cc(Cl)cc([N+](=O)[O-])c1', 'N#C[C@H]1C(N)=N[C@H]([C@H]2CC[NH2+]C2)c2cc(Br)ccc21', 'Clc1ccc(Br)cc1Cc1nnc(-c2ccc(Cl)cn2)n1N', 'Clc1cc(F)c(F)cc1C[C@H]1CCO[C@@]2(CCOC2)C1', 'Clc1cc(Br)ccc1O[C@@H]1CCOC2(CCC2)C1', 'Oc1cccc(Br)c1Oc1ccccc1C(F)(F)F', 'OC[C@H]1CC[C@]2(CCc3c(F)cc(Br)cc3O2)C1', 'Clc1ccc(CCOC(F)F)c(Br)c1', 'O=[N+]([O-])c1c(Cl)nnc2onc(-c3cccc([N+](=O)[O-])c3)c12', 'Oc1ccccc1/C=N/NC(=O)c1ccc([N+](=O)[O-])cc1', 'Clc1ccc(Br)c(Nc2nnc3c(O)nc(SCC(F)(F)F)n3n2)c1', 'Oc1cccc(Nc2cc(Cl)ccc2Cl)c1', 'Clc1ccccc1C(=O)Oc1cccc(Cl)c1', 'Clc1cccc(Nc2cc(Cl)ccc2O)c1', 'Clc1cccc(/N=N\\c2cccc(Br)c2)c1', 'Oc1cc(F)nc(Nc2ccc(C(F)(F)F)cc2)c1', 'ClCc1cc(F)cc(N2CCNCC2)c1', 'Clc1nnn(-c2nccc(Cl)c2Cl)c1NS(=O)(=O)c1cccc(F)c1', 'OC1=Nc2ccccc2C(c2ccccc2)(c2ccc(Br)cc2)C1', 'Clc1ccc(Br)cc1[C@@H]1COC2(CCCC2)C1', 'Clc1ccc(CO)c(C(=O)N2CCCCC2)c1', 'Clc1cc(Br)cnc1[C@H]1CCCN1S(=O)(=O)c1cccs1', 'Clc1ccc(Br)cc1[C@@H]1OCCc2ccccc21', 'Clc1cc(Br)c(Cc2ccccc2)c2c1COC2', 'NS(=O)(=O)/N=C/C(=O)Nc1cccc(Br)c1', 'O=[N+]([O-])c1ccnc2ccc([N+](=O)[O-])cc12', 'Oc1ccc(F)c(/N=C2\\S[C@H](c3ccco3)N=N2)c1', 'Clc1cc(Nc2nc(C3CC3)no2)c2ncccc2c1F', 'O=C(/C(c1cccs1)SC1=NC=N1)c1ccc(Cl)cc1', 'O=C(c1c[nH]cn1)[C@H]1[C@H](c2cccc(Cl)c2)CCC1=O', 'O=[N+]([O-])c1c([O-])sc2scnc12', 'O=[N+]([O-])c1c(-c2n[nH]c([O-])n2)ccc2nsnc12', 'Oc1cccc(-n2c(Sc3ccccc3)n[nH]c2=O)c1', 'Oc1cc(-n2nnc(C(F)(F)F)cc2=O)ccc1N1CCOCC1', 'Clc1nnnc(S(=O)(=O)N2CCOCC2)c1-n1cccc1', 'Oc1cccc(-c2c[nH]nc2CN2CCNC(=O)[C@H]2c2cccs2)c1', 'Clc1ccccc1-c1ncc(F)cc1C[S@@](=O)c1cscc1C(F)(F)F', 'ClCc1csc(SCc2noc(-c3ccccc3O)n2)n1', 'ClCc1cc(CCO)ccc1F', 'N#C/C([O-])=C(\\CC(=O)c1ccc(F)cc1)c1cc(C#CCCO)ccc1O', 'Clc1cc(Br)cc2c1OCCN1C[C@@H]1C[C@@H]2C(=O)[O-]', 'Oc1cccc(Nc2snc3ccccc23)c1F', 'Cc1snnc1SCC(=O)Nc1cccc([N-]S(C)(=O)=O)c1', 'O=C(CSCNS(=O)(=O)c1ccc(Cl)cc1)NNC(=O)OCc1ccccc1', 'O=[N+]([O-])c1ccnc2ccc(S(=O)(=O)C[C@H](O)c3cccs3)cc12', 'Nc1[nH]nc(COc2cccc(Br)c2)c1N', 'Clc1ccsc1C[N-]C(=O)c1ccc(Cl)s1', 'Clc1cc(/N=N/S(=O)(=O)c2ccc(C(F)(F)F)cc2)cc(Cl)c1Cl', 'O=C(Cc1noc([O-])n1)N[C@H]1c2ccccc2C[C@@H]1O', 'O=C(COc1ccc2cc([N+](=O)[O-])ccc2c1)Nc1cccc(Cl)c1', 'Clc1cc(/C=C(\\C#N)c2cccc(Cl)c2)cc(Cl)c1Cl', 'Oc1cc(F)c([O-])c(C(=O)O[C@@H]2CCCCC2(Cl)F)c1Cl', 'Oc1ccc(/C=C(\\[O-])/C([O-])=N\\c2ncccn2)c(O)c1', 'Oc1cc(-n2nc([C@@H](Cl)C(=O)[O-])ccc2=O)co1', 'O=[N+]([O-])c1c(F)cc(Nc2ncccn2)o1', 'Clc1ccccc1Oc1cc(F)c(F)cc1F', 'Clc1ccccc1Oc1cc(F)c(Cl)cc1Cl', 'Clc1cc(Br)c2nc(OCc3cscn3)[nH]c(=O)c2c1', 'Clc1ccccc1Nc1cccc(C(=O)Oc2ccc(C(F)(F)F)cn2)c1', 'Clc1ccc(Br)c(NC2CCCCC2)c1', 'Oc1ccc([N+](=O)COc2ccccc2Cl)cc1O', 'O=C(CNc1c(F)cc(F)cc1Cl)Nc1ccc(F)c(F)c1', 'Clc1ccc(Br)c([C@@H]2OCCc3ccc(F)cc32)c1', 'Oc1cc(Br)cs1\n\n\n\n\n\n\n', 'Oc1cc(Br)cs1\n\n\n\n\n\n', 'Oc1cc(Br)cs1\n\n\n\n\n', 'Oc1cc(Br)cs1\n\n\n\n', 'Oc1cc(Br)cs1\n\n\n', 'Oc1cc(Br)cs1\n\n', 'Oc1cc(Br)cs1\n', 'Oc1cc(Br)cs1', 'Oc1cc(Br)cs1', 'Oc1cc(Br)cs1', 'Cc1noc(C)c1S(=O)(=O)/N=C(\\[O-])[C@]1(C)CC1(Cl)Cl', 'Oc1ccc(F)c(C/C(Cl)=C2\\CCCCN2)c1', 'O=C(CSCc1ccccc1)Nc1ccc(S(=O)(=O)C(F)F)cc1', 'N#Cc1c(/C=C\\C(=O)c2cccc(F)c2)cn[nH]/c1=N\\OCCc1ccccc1', 'Oc1cccc(-c2csnn2)c1O', 'Oc1cccc(Nc2c(C(F)(F)F)ncnc2/N=C/c2ccccc2Cl)c1', 'NS(=O)(=O)/C(C#N)=C\\c1cccc(Cl)c1', 'ClCc1cccc(C(=O)Oc2ccc3c(c2)CCCN3)c1', 'Clc1nnnn1-c1c(C(F)(F)F)cc(C(=O)[O-])c2ccccc12', 'Clc1cccc([S@@]c2ncnc(Nc3ccc(Br)cc3Cl)c2[O-])c1', 'Oc1ccc([N+](=O)[O-])cc1Sc1ccccc1C(=O)Nc1nc(Cn2ccc(C(F)(F)F)c2)n[nH]1', 'Clc1nnnc(SCc2cc(F)ccc2F)n1', 'Oc1ccccc1/N=N\\c1ccccc1Br', 'OC1=Nc2ccccc2C[C@@H](c2ccccc2)C1', 'O=[N+]([O-])c1c(O)sc2c1nnn2-c1ccccc1', 'Oc1ccc(/C=N\\c2ccccc2Cl)c(Cl)c1', 'Clc1ccccc1Oc1cc(O)ccc1F', 'Clc1ccccc1Oc1cc(F)ccc1O', 'Clc1ccccc1Oc1cc(F)ccc1O', 'Clc1cccc2c1Nc1nnc(SCc3ccc(Br)s3)nc12', 'Clc1nncnc1[N-]Sc1ccncc1', 'O=[N+]([O-])c1cc(Br)c2nccc(=O)n2c1N', 'Oc1cccc(SCn2cnc3ccccc3c2=O)c1', 'O=[N+]([O-])c1c([O-])[nH]c(=O)c2ccc(F)cc12', 'COC[C@@H]1OC(=O)c2cc(Br)ccc21', 'Oc1ccc([N+](=O)C=[N+]2NC(=O)N(c3ccccc3)c3ccccc32)cc1', 'N#C/C(=C\\c1cc(Cl)ccc1OCCO)C(F)(F)F', 'Clc1ccc([C@@H](Br)c2cc(F)ccc2F)cc1', 'Clc1ccc(Br)cc1Cc1ccc(Cl)cc1F', 'Clc1ccc(Br)cc1-c1ccc(Cl)cc1F', 'Clc1ccc(Br)cc1-c1cc(F)ccc1F', 'Clc1cc(NSc2nc3ccccc3[nH]2)ncn1', 'Clc1nnnn1/C([O-])=N/c1ccc([N+](=O)[O-])cc1', 'Clc1cccc(-c2ncnn2Cc2ccco2)c1', 'Clc1nnnc(S[C@@H]2CCCOc3ccc(OC(F)(F)F)cc32)c1Br', 'OC1=NC(SCC(=O)Nc2cccc(Cl)c2)=NN1', 'Clc1nncnc1Nc1ccc([N+](=O)[O-])cn1', 'Oc1cc(Br)c2sncc2c1F', 'Clc1nncc(Nc2cccc(C(=O)[O-])c2)n1', 'COc1nnn(-c2cc(N)ccc2Sc2ncccn2)c1C', 'O=C([O-])c1[nH]nc(-c2cccc([N+](=O)[O-])c2)c1-c1ccccc1C(F)(F)F', 'Clc1nnnn1-c1cc([N+](=O)[O-])ccc1O', 'OCc1nnsc1-c1ccc2c(c1)OCO2', 'ClC[C@@H]1OCC[C@@H]1c1ccc(F)cc1', 'ClC(c1ccc([N+](=O)[O-])cc1)[C@H]1CCOC2(CCC2)C1', 'Clc1ccc([C@@H](C)[NH2+][C@H]2CCCNC2=O)cc1', 'Oc1cc(O)c2nc(NCc3cccnc3)[nH]c(=O)c2c1', 'ClC[C@H]1O[C@H]2[C@@H](c3ccccc3)N=C12', 'C=C1c2ccc(Cl)cc2CCN(C(=O)[C@H]2CC=CCC2)C1', 'Clc1cccc(/C(=C/Nc2ccc(F)c(F)c2)C(=O)[O-])c1', 'Oc1ccc(F)cc1-c1cccc(F)c1O', 'C[C@@H](c1nc[nH]n1)n1ccc(NC(=O)c2ccc(C#N)cc2)n1', 'Oc1ccc([N+](=O)[O-])c(-c2csc(N3CCCCC3)n2)c1', 'Clc1ccccc1[C@H](N)C(=O)[O-]', 'Clc1cc(C(F)(F)F)nc(-c2cccc(F)c2Cl)n1', 'ClCc1nnsc1[N+](=O)[O-]', 'O=[N+]([O-])c1ccn(-c2cc(C[NH3+])ncc2F)c1', 'O[C@@H](c1ccc2c(c1)CCC2)C(F)(F)F', 'O=[N+]([O-])c1cc(Br)ccc1Oc1ccc(Cl)cc1', 'Clc1ccc(-c2csc3[nH]c([C@@H]4CCCNC4=O)nc23)cc1', 'Clc1ccsc1SC1CCN(c2ncnc3[nH]c[nH+]c23)CC1', 'Clc1cccc(-c2ccnc(Sc3cccc(C(F)(F)F)c3)n2)c1', 'O=C(c1ccc[nH]c1=O)N1N=C(c2cccs2)C[C@H]1c1cccnc1', 'Clc1ccc(-c2cc(F)ccc2F)cc1', 'Clc1ccc(-c2cc(F)ccc2F)cc1', 'Clc1ccc(-c2cc(Cl)ccc2Cl)cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(Oc2ccc(Cl)cc2Br)c1C(F)(F)F', 'OCSc1nc(-c2ccco2)nc2cc(Cl)ccc12', 'ClCc1cc(C#N)ccc1Oc1ccccc1', 'Oc1cccc(-c2cc(F)c(Br)c(Cl)c2)c1', 'Clc1cc(N2CCc3sccc3C2)ccc1C(F)(F)F', 'Cc1ccc2nc(NC(=O)c3ccc(C(=O)[O-])cc3)sc2c1C', 'Oc1ccc([N+](=O)[O-])c(Nc2cccc(Cl)c2)c1', 'Clc1cc(F)c(N/N=C/c2ccccc2O)cc1', 'Clc1cc(Br)cc(Cl)c1OCC[NH3+]', 'N[C@@H]1NC(=O)/C(=C/c2ccco2)S1', 'O=C(CSCNOc1ccccc1Cl)Nc1ccc(O)cc1', 'O=C(/C=C/c1cccs1)NNC(=O)Oc1ccccc1C(F)(F)F', 'COc1ncncc1C[S@@](=O)Cc1nc(-c2cccs2)no1', 'NS(=O)(=O)/C=C/COc1cccc(F)c1[N+](=O)[O-]', 'Oc1cc(Br)c(/C=N/Nc2ccccc2)cc1F', 'Clc1ccc(CO)cc1O', 'Clc1cc(/C=N\\c2ccc(F)cc2F)ccc1[N+](=O)[O-]', 'Oc1ccc(F)c(S(=O)(=O)[O-])c1', 'Oc1ccc([N+](=O)CN(C(=O)c2cccc([N+](=O)[O-])c2)c2ccccc2)cc1', 'Oc1ccc(F)c(Sc2n[nH]cc2C(=O)NCCc2cccc(O)c2)c1', 'Clc1ccc(CN2C(=O)CNC2=O)cc1', 'C[C@@H](OC(=O)c1ccc(OCC#N)cc1)C(F)(F)F', 'Clc1ccc(Br)c(NN2C(=O)CSC2=O)c1', 'Oc1ccc([N+](=O)c2ccccc2Cl)c(I)c1', 'Clc1ccccc1Oc1cc(F)c(I)cc1O', 'Clc1ccccc1Oc1cc(Br)c([O-])c(Br)c1', 'Cc1sncc1-c1c(C)nc2[nH]c(C(C)C)nc2c1[O-]', 'Oc1cc(F)c(Cl)cc1N1CC[C@H](N2CCCC2)C1', 'Nc1nnc(Nc2cccc(Oc3ccccc3)c2)s1', 'O=[N+]([O-])c1c(Cl)nc(Cl)cc1Br', 'ClCc1nnsc1-c1ccc(Cl)c(Cl)c1O', 'Oc1ccccc1C1NC[C@@H](c2cccs2)Cc2cccnc21', 'Clc1nnnc(-n2nc([O-])nc2-c2ccc(Cl)s2)n1', 'O=C(C1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1)N1CCC(c2nc(-c3cccnc3)c[nH]c2=O)CC1', 'Oc1ccc(/N=c2/sccc(=O)n(Cc3ccc4c(c3)OCO4)c2=O)cc1', 'O=C(CNc1ccc(Cl)cc1[N+](=O)[O-])Nc1ccc(F)c(F)c1', 'CC(C)=CC(=O)C[C@@H]1C(=O)NCCN1C(=O)c1cccc(Cl)c1', 'Clc1ccc(Br)c(-n2nnnc2Sc2ccccc2)n1', 'O=[N+]([O-])c1ccncc1Sc1ncc[nH]1', 'O=[N+]([O-])c1c(Cl)nsc1Nc1ccc(Cl)cc1F', 'Clc1cccc(Nc2cc(-c3cccs3)no2)c1', 'Clc1cc(Cl)cnc1N/N=C1\\N=C(c2ccco2)S1', 'N#C[C@@H]1C[NH+](Cc2ccc(F)cc2)C=C1c1ccc(Br)cc1O', 'Clc1cc(Cl)nn1/N/c1cccc([N+](=O)[O-])c1', 'Nc1cc([O-])nc([C@@H]2CCCN(c3ncnc4ccsc34)C2)n1', 'COc1nc(C)cc(C)c1NC(=O)C(=O)Nc1ccc2[nH]c(=O)[nH]c2c1', 'Clc1ccc(Br)cc1/N=N\\c1ccc(Cl)cc1[N+](=O)[O-]', 'CN(C[C@H](C[NH3+])c1ccc([N+](=O)[O-])cc1)C(C)(C)C(=O)NC1CC1', 'Oc1ccc([N+](=O)c2cc(Cc3ccc(F)cc3)cs2)cc1', 'COc1ncc2nc(NC(=O)Nc3cc(C(C)(C)C)cs3)nn2c1', 'Clc1cc(N2CCc3ccccc32)cc([N+](=O)[O-])c1', 'Clc1cc(F)c(C[C@@H]2CCC[C@@H]2O)cc1', 'NS(=O)(=O)/C(C#Cc1ccccc1)c1ccc(Cl)cc1', 'Oc1ccc(O[C@@H](C(=O)[O-])c2cccc(Cl)c2)cc1', 'Oc1cccc2cc([C@@H]3Nc4ncccc4C(=O)O3)c(O)cc1O[C@H]2c1cccc(Cl)c1', 'Clc1ccc(Cc2cc(F)c(F)c(Cl)c2)cc1Br', 'O=C(c1cn[nH]n1)N(c1nncs1)c1ncccn1', 'Oc1ccc(/N=C/c2cccc(Cl)c2C)cc1Cl', 'Nc1nonc1C(=O)Nc1sccc1C(F)(F)F', 'COc1nc2snnc12', 'NS(=O)(=O)Cc1ncsc1CO', 'O=C(CNc1nnccn1)Nc1c(F)cccc1F', 'Oc1ccccc1-c1c(Cl)cc(Cl)cc1CBr', 'Clc1cc(Br)c([C@H](O)c2ccccc2)cc1F', 'ClCc1csc(Oc2cccc(F)c2)n1', 'O=[N+]([O-])c1c([O-])nnnc1Sc1ccccc1', 'Clc1ccc(Br)cc1Sc1ncccc1OCc1ccoc1', 'Clc1cc(/C=N\\O)ccc1F', 'Oc1ccc(Oc2nc3ccsc3c(=O)o2)cc1Cl', 'Clc1ccccc1Oc1cc([N+](=O)[O-])cc2c1OCOC2', 'O[C@@H](Sc1nnns1)C(=O)Nc1ccc(Cl)cc1', 'ClCc1cc(Br)c(Br)s1', 'Oc1cc(Br)c(OCc2cc(F)cc(F)c2)c(O)c1', 'Oc1ccc(F)c(S(=O)(=O)c2cccc(F)c2F)c1', 'Clc1ccsc1NC1=NS(=O)(=O)c2ccccc21', 'Clc1nnc([S-])s1', 'Clc1ccccc1NCCS(=O)(=O)c1ccccc1', 'Clc1ccc(Br)cc1NCCS(=O)(=O)c1ccccc1', 'Clc1cc(F)c(I)cc1I', 'Clc1ccc(-c2nonc2C/[NH+]=C2\\SC3=C(CCCC3=O)NC2=O)cc1', 'O=[N+]([O-])c1c(-c2nnn[n-]2)sc2cc(Br)ccc12', 'Clc1ccc(C(N)=[NH2+])cc1', 'Clc1ccc(C(N)=O)cc1', 'Clc1ccc(C(N)=O)cc1', 'O[C@@H](c1cc(Br)ccc1F)c1cccc(F)c1', 'Oc1ccc(F)c(Oc2ccccc2C(=O)[O-])c1', 'OC[C@H](O)c1ccc(C(F)F)cc1Br', 'Clc1ccc(Br)cc1N1/C=Nc2ccc(Cl)cc2C1', 'Oc1cccc(Br)c1Oc1ccc(F)cc1Cl', 'Oc1ccc(F)c([C@@H]2O[C@@H]3CCC[C@H]2[C@@H](O)[C@@H]3O)c1', 'Clc1ccc(C(N)=NC2=NS(=O)(=O)c3ccccc32)cc1', 'Clc1cc(/C=N/NC(=S)NC2CC2)nc2cccnc12', 'NS(=O)(=O)/C=c1/sc[n+]([O-])c2ccccc12', 'Oc1cccc2cc(C(=O)Nc3ccc(F)cc3)cn-2/c([O-])c1', 'Clc1nnnn1CC[C@@H]1CSc2ccccc21', 'Clc1cccc(-c2n[nH]/c(=C/c3ncccc3Cl)o2)c1', 'O=C(c1cc(F)c2ccccc2c1Cl)Nc1ccc2c(c1)OCO2', 'Clc1ccccc1N1N=Cc2ccc(Cl)cc2C1', 'N#Cc1c(Cl)cc(Cl)cc1C(=O)NCc1cccc(Br)c1', 'N#C[C@@H]1COCCCN1Cc1ccc(Br)s1', 'Clc1ccc(Br)cc1OS(=O)(=O)C(F)(F)F', 'O=C([C@H](Nc1ccccc1)C(F)(F)F)Nc1ccccc1C#N', 'O=C(/C=C/O)c1ccc(Cl)cc1', 'O=[N+]([O-])c1c(Br)scc1Oc1cccc2cccnc12', 'Clc1cccc2cc(Br)c(/C=C/C(=O)[O-])cc12', 'Clc1ccc(Br)cc1C(F)(F)C1CC1', 'Oc1ccccc1[C@@H](c1cccnc1)N1CCN(S(=O)(=O)c2ccc(Cl)cc2F)CC1', 'Oc1cccc(Br)c1C(F)(F)F', 'Oc1cccc(Br)c1C(F)(F)F', 'Clc1ccccc1Nc1cc([N+](=O)[O-])c(Cl)cc1Cl', 'Clc1ccsc1/C([O-])=N/Oc1cncc(Cl)c1', 'Clc1nnnn1/N=C\\c1cc([N+](=O)[O-])ccc1O', 'Clc1nnnn1/N=C/c1ccc([N+](=O)[O-])cc1O', 'NS(=O)(=O)Nc1cc(Br)ccc1C(F)(F)F', 'Clc1ccc(-c2nonc2[S-])s1', 'Clc1ccc(-c2nonc2[S-])s1', 'Clc1cc(CCl)cc(I)c1O', 'Oc1cc(C=O)c2sccc2c(=O)c1Br', 'O=C(Cc1ncon1)Nc1ncc(Br)cc1Cl', 'Oc1cc(Br)c2sccc2c1Oc1c(F)cc(F)c(F)c1F', 'Clc1cc(NSc2nc3ccccc3s2)ccc1Br', 'N#C/C(=C\\c1ccc(-c2cccc([N+](=O)[O-])c2)o1)C(=O)NCCc1ccccc1', 'O=[N+]([O-])c1c(Cl)nn(-c2ccc([N+](=O)[O-])cc2)c1[O-]', 'Oc1ccccc1-c1c(F)cc(Cl)cc1F', 'O=C(CNc1cccc(C(F)(F)F)c1)NC(=O)c1ccc(Cl)nc1', 'Clc1ccccc1Oc1cc(Br)c(F)c(F)c1F', 'Clc1ccsc1S(=O)(=O)N1CCc2nccnc2C1', 'O[C@@H](S[C@@H](F)c1cccnc1)c1ccccc1F', 'Clc1ccccc1NS(=O)(=O)c1cc(Cl)cc(Cl)c1', 'O=C(c1ccsc1)c1ccc(Cl)c(C(F)(F)F)c1', 'Oc1cccc([C@H]2[C@H]3[NH+]=c4ccccc4=C3CC[C@@H]2O)c1', 'Oc1cccc(Br)c1S(=O)(=O)NCc1cccs1', 'ClCc1csc([N-]Sc2ccccc2)n1', 'O=[N+]([O-])c1c([O-])nsc1Nc1cccc(I)c1', 'Clc1ccc(Br)cc1[C@@H]1CCCN1C(=O)c1ccc2c(c1)CCC2', 'Clc1cc(Cl)ccc1N1CCc2cc(Cl)ccc21', 'Clc1ccc(CO)c([C@H]2CSCCO2)c1', 'NS(=O)(=O)/C(C#N)=C/c1ccc(-c2ccc(Cl)cc2)o1', 'Clc1cc(NC2(c3ccc(F)cc3)CCOCC2)cs1', 'Oc1ccc(/C=C2/C(=O)N(c3ccccc3)SC2=S)cn1', 'Oc1ccc(F)c(C[C@@H]2CCC[NH2+]2)c1', 'Oc1ccc(F)c(-c2cccc(F)c2F)c1', 'Clc1cc(Cl)cc(Cl)c1C[NH+]1CCC(O)[C@H](O)C1', 'N#Cc1nsnc1CCC(=O)NC1CCc2cccc(F)c21', 'Oc1cccc(Br)c1O/N=C1/C(=O)C2=C(c3ccccc3)CS[C@H]1N2', 'Clc1ccc(Br)c(C[C@@H]2CCC[NH2+]2)c1', 'Clc1cccc(/N=N/c2ccc(F)c(F)c2)c1', 'NS(=O)(=O)Cc1csc(NC(=O)c2cccc([N+](=O)[O-])c2)n1', 'O=C([O-])C1=NO[C@@H]2NC(=O)N(c3ccccc3)/C1=N/OCCC2', 'Clc1ccc(Br)cc1[C@@H](O)COC(F)(F)F', 'Clc1ccc(Br)cc1O[C@H](CO)C(F)(F)F', 'NS(=O)(=O)/C=C\\c1cc(Cl)cc(Cl)c1', 'O=C(/C=C1/N[C@H](c2ccccc2)SC1=S)c1ccc(F)cc1', 'O=[N+]([O-])c1c(F)cccc1Nc1ccc(Cl)cc1[N+](=O)[O-]', 'N#C[C@@H]1C(=O)N(CCC(=O)Nc2cc(F)ccc2F)C(=O)N(C(C)(C)C)C1=O', 'Clc1ccsc1-c1nnc(CSc2ncccn2)[nH]1', 'Oc1cccc(OC(F)F)c1NC1=NN=C(c2ccccc2)C1', 'Clc1cc(/N=C/C(=N/O)c2sccc2C(=O)[O-])ccc1O', 'COCc1c(C)sc2ncnc(N/C(=N/O)S(C)(=O)=O)c12', 'N=C1[C@H]2CCCCN1Cc1cc(F)ccc12', 'Clc1ccc(Br)c(Sc2nc(=O)n3ccsc3n2)c1', 'Oc1cc(Br)cc2ccccc12', 'O=[N+]([O-])c1c([O-])sc2nc([O-])[nH]c12', 'O=C(CS(=O)(=O)Cc1cccc(Cl)c1)Nc1ccccc1C(=O)[O-]', 'Clc1ccccc1Cc1cc(F)ccc1O', 'Clc1cc(F)ccc1[C@H](O)c1ccccc1', 'NS(=O)(=O)/N=C/Cc1ccccc1Br', 'COC(=O)[C@]1(F)CCN(C(=O)c2cc(F)c(F)cc2Br)C1', 'ClC[C@@H]1C(C#N)C=CC[C@@]1(O)c1ccccc1', 'O=[N+]([O-])c1cc(Cl)ccc1-c1nc(C(F)(F)F)cs1', 'O=C(CSCc1cccc(Br)c1Cl)Nc1ccccc1C(=O)NCc1ccccc1', 'Oc1cc(O)c(N[C@@H]2CC[C@H]([NH3+])C2)cc1Cl', 'Clc1cc(Cl)c(CN2CCOCC2)cc1O', 'Clc1cccc(Br)c1CNc1cc([N+](=O)[O-])ccc1O', 'Clc1ccc(Br)c(Sc2nnnn2-c2ccccc2)c1', 'Clc1cc(Cl)c(N2CCN([C@@H]3[C@@H](O)NC2=O)CC3)cc1Cl', 'ClC[C@@H]1Oc2cc(F)ccc2C[C@@H]1F', 'Oc1ccc(/C=N/O)cc1Br', 'Clc1cccc(F)c1-c1cccc(C(F)(F)F)c1', 'O=C(CNCc1c(O)cc([N+](=O)[O-])cc1F)Nc1cccc(O)c1', 'Clc1ccccc1Oc1cc(F)c(F)c(F)c1F', 'CNC(=O)Nc1cccc(C(=O)N2CCCc3cc(C)ccc32)c1', 'Clc1cccc(Nc2cn[n+]([O-])cc2O[C@@H]2CCCCS2)c1', 'Cc1ccsc1[C@@H](N)Cc1ccc(Cl)c(F)c1', 'NS(=O)(=O)/N=c1\\[nH]c(-c2cccs2)nc2cc(C(=O)[O-])ccc12', 'O=[N+]([O-])c1cc([C@H]2CC(=O)N(c3ccc(Cl)cc3)C2=O)ccc1[O-]', 'O=C(C1=Nc2sccc2[C@@H]1O)[C@H]1CSC(=S)N1c1ccccc1', 'Clc1cccc(Nc2cc([N+](=O)[O-])ccc2Br)c1', 'Clc1cc(Br)c([N+](=O)[O-])cc1Sc1ncccn1', 'Clc1ccc(Br)cc1N1CCN(C(=O)C(F)(F)F)CC1', 'CCS(=O)(=O)c1ccc(NC(=O)c2cccc(C#N)c2)cc1', 'O=C(c1ccc(F)cc1F)C(=O)Nc1ccc(C(=O)[O-])cc1', 'Nc1c(Cl)c2c(cc1Cl)c1c([nH]c1=O)CCCC2', 'CCc1c(C)sc(NC(=O)c2c3ncn(CC(C)C)c3sc2C)c1C#N', 'Cc1ncsc1C(=O)Nc1ccccn1', 'Clc1ccc(Br)cc1N1C(=O)/C(=C\\c2cccnc2)SC1=S', 'NS(=O)(=O)C[C@@H](F)c1cccnc1', 'CN=C1/C(=N/C(=O)[C@@H](C#N)c2ccc(Cl)cc2)S/C1=N\\c1cccc(F)c1', 'Clc1ccc(-c2cc(Br)ccc2N2CCCC2)cc1', 'Oc1cccc(C(F)(Cl)NNC(=O)c2csc(Br)c2)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(I)c1-c1ccc([N+](=O)[O-])cc1', 'Oc1ccc(/N=c2/sc3c(s2)CCN3C(F)(F)F)c(F)c1', 'Oc1cc(Br)c([C@@H]2C[NH2+]CCC[C@H]2O)cc1F', 'O=C(c1cc(F)ccc1F)N1CCC[C@@H]1[C@@H]1CCCC[C@@H]1O', 'Clc1ccc(-c2nn([C@@H]3CCO[C@H]2c2ccccc2F)C3)cc1', 'ClC(=S)N\\c1ccc(F)c(-c2cscn2)n1', 'Clc1nnnn1c1cccc(C(F)(F)F)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1Nc1cccc(Cl)c1', 'O=[N+]([O-])c1c(F)cc(NCc2cccc(O)c2)cc1Cl', 'O=[N+]([O-])c1ccc(CN2CCOc3ccc(Cl)cc32)c(Cl)c1', 'O=[N+]([O-])c1ccc2nc(-c3cccc([N+](=O)[O-])c3)[nH]c(=O)c2c1', 'Clc1cc(Br)c2nc(O)sc2c1', 'NS(=O)(=O)Cc1noc(-c2sncc2-c2ccccc2)n1', 'O=C([O-])/C(=N\\O)c1ccccc1Br', 'Clc1ccc(C(F)(F)F)c(F)c1Cl', 'Clc1ccc(Br)c(F)c1C(F)(F)F', 'Clc1ccc(-c2cc(Br)c3c(c2)OCCO3)cc1', 'Clc1ccsc1NSc1ccc(Br)cn1', 'Clc1ccc(CO)c(SCc2nc3ncccc3o2)c1', 'Clc1ccc(C(=O)NNC(=O)N2CCCCCC2)cc1F', 'O=C([C@H](Cl)Br)c1ccc(Cl)cc1', 'Clc1cccc(F)c1-c1ccc(Br)c(C(=O)[O-])c1', 'Clc1cc(/N=C/C(=O)N[NH+]2CCOCC2)cs1', 'Clc1cc(NSc2nnnn2Cc2ccccc2)cs1', 'Clc1ccsc1/C=N/NC1=NS(=O)(=O)c2ccccc21', 'O=[N+]([O-])c1c(F)cccc1[C@@H](O)c1cc(F)ccc1F', 'Clc1ccc(Br)cc1-c1ccs/c(=O)c1C(F)(F)F', 'NS(=O)(=O)CCNc1ccc(Oc2cccc(C#N)c2)cc1', 'Oc1ccccc1Cc1cc(F)ccc1Br', 'Oc1ccc([N+](=O)c2ccco2)cc1O', 'Clc1cccc([S@@]C2CCN(S(=O)(=O)c3cccnc3)CC2)c1', 'Clc1ccccc1N1NC(c2ccccc2)=NC1', 'Clc1ccc(Br)c(CCl)c1', 'Clc1cc(CCl)ccc1Br', 'Clc1cc(CCl)ccc1Br', 'Clc1ccc(C2(c3ccccc3F)C[NH2+]C2)cc1', 'Clc1cc(F)cc(N/N=C2\\N=NC3=C2CCc2ccccc23)c1', 'Oc1cc(-n2nnnc2O)cs1', 'Oc1ccc([C@H]2CC[NH2+]C[C@@H]2C(=O)[O-])cc1F', 'O[C@@H]1[C@@H](O)[C@@H](O)[C@@H]1Cc1cc(Cl)ccc1Cl', 'O=C([C@H]1Cc2cc(Cl)ccc2O1)N1CCC(O)CC1', 'ClCc1cc(O)cc(F)c1I', 'Clc1ccsc1S(=O)(=O)Nc1ccc(C(F)(F)F)cc1', 'C[C@@H](c1nc(-c2cccnc2)n[nH]1)S(=O)(=O)c1ccc([N+](=O)[O-])cc1', 'Clc1cc(NSc2ccc3nccnc3n2)ncn1', 'Clc1cc(Cl)cc2c1NN(c1ccccc1C(F)(F)F)NC2', 'Clc1cccc2cc(CNc3cccnc3Cl)sc12', 'O=[N+]([O-])c1c(F)cc(Cl)cc1OCC(F)(F)F', 'Cc1nonc1CCC(=O)NCc1cc(-c2ccc(C#N)cc2)on1', 'Oc1ccc(F)c([C@@H]2CCCN2c2cccc(C(F)(F)F)c2)c1', 'O=[N+]([O-])c1ccc2cc(=O)oc(=O)c(-c3cccc([N+](=O)[O-])c3)c2c1', 'C=C(Cl)C[NH+]1CCc2c(F)cc(F)cc2[C@H]1C', 'Clc1ccccc1Oc1cc([N+](=O)[O-])ccc1O', 'Clc1ccccc1Oc1cc([N+](=O)[O-])ccc1O', 'Clc1ccccc1Oc1cc([N+](=O)[O-])ccc1Cl', 'Clc1cccc(Nc2cc([N+](=O)[O-])ccc2Cl)c1', 'O=C([C@H](C#N)c1cccs1)N1CCC[C@@H](c2ncccn2)C1', 'O=C(CSCc1csnn1)NCc1ccc(-c2nc3ccccc3o2)cc1', 'Oc1cc(Br)c([O-])c(C#CCOc2ccccc2)c1', 'Clc1ccc(Br)cc1/N=C/c1ccc2c(c1)OCO2', 'Oc1cccc(C(=O)Nc2nc(-c3ccc(F)cc3F)cs2)c1', 'Oc1ccc(Oc2cc(C(=O)[O-])cc([N+](=O)[O-])c2)cc1', 'Clc1cc(NSc2ccncn2)ncc1Cl', 'Clc1cc(/N=C\\C2=CCNC2)c2ccccc2c1Cl', 'Clc1ccsc1[C@H]1[NH+]=c2ccc(O)cc2=[NH+]C1=O', 'Clc1ccc(Br)cc1/N=C1/NNc2ccc([N+](=O)[O-])cc21', 'Oc1cc(F)c(Oc2ccc(Cl)cc2F)c(F)c1', 'Clc1ccsc1[C@@H](F)C(=O)[O-]', 'OC[C@@H](Sc1nncs1)C(=O)Nc1cccc2ccccc12', 'NS(=O)(=O)/C=C/c1ccc(F)cc1', 'Oc1ccc(F)c(/C=C2\\N=c3ccccc3=[NH+][C@@H]2c2cccc([N+](=O)[O-])c2C(F)(F)F)c1', 'Clc1ccccc1C1=CCN=CN1', 'Clc1cc(/C=N/N2CCCCC2)c2c(c1)OCO2', 'Oc1cccc(SCn2nc3cccnc3n2)c1', 'Oc1cccc2cc(O[C@H]3CCOC3)cc(C(F)(F)F)c12', 'NS(=O)(=O)/C=C/COc1ccc2cccnc2c1', 'Nc1c(F)c(O)c2ccccc2c1OC(=O)CSc1nccs1', 'O=C(CNc1nn2nc(-c3cccc(F)c3)oc2c1Cl)Nc1cc([N+](=O)[O-])ccc1F', 'Clc1ccc(Br)c(/N=N/c2ccccc2)c1', 'O=[N+]([O-])c1c(Cl)nsc1Cl', 'O=[N+]([O-])c1c(Cl)nsc1Cl', 'O=[N+]([O-])c1c(Cl)nc(Br)s1', 'Cc1noc2nc(S[C@@H](C)C(=O)c3ccccc3)sc12', 'Clc1nnn(-c2ncnc3sc4c(c23)CCC4)n1', 'O=[N+]([O-])c1cc(Br)c(-c2cccc(Br)c2)cc1O', 'Clc1ccc(Br)c(Cl)c1\n', 'Clc1ccc(Br)c(Cl)c1', 'Clc1ccc(Br)c(Cl)c1', 'Clc1ccc(Br)c(Cl)c1', 'Clc1cc(Cl)ccc1I', 'O[C@@H]1c2cc(Cl)ccc2N[C@H]1CC(F)(F)F', 'Clc1ccc([C@@H](Cl)c2ccc3c(c2)OCCO3)cc1F', 'ClC(c1cc2c(cc1Cl)OCCO2)c1ccc(F)cc1F', 'Clc1ccccc1N1N=CN(S(=O)(=O)c2ccc(Cl)c(Cl)c2)CC1', 'O=C(/N=C\\N=c1\\scc(-c2ccc([N+](=O)[O-])cc2)[nH]1)c1ccc[nH]1', 'Oc1ccc(F)cc1O[C@@H](CO)C(F)(F)F', 'NS(=O)(=O)N1Cc2ccc(OC(=O)c3cc(C#N)ccc3Cl)cc2C1', 'O=C(/C=C/c1c(F)cccc1Cl)Nc1ccc([N+](=O)[O-])cc1', 'N#Cc1nc(-c2ccsc2)nc2c1CC[C@@H](c1ccccc1)C2', 'O=[N+]([O-])c1c([N+](=O)[O-])cc([N+](=O)[O-])c2ccccc12', 'Clc1ccccc1OCc1cc(=O)n2c(n1)SCCN2', 'Nc1c(F)c(Br)ccc1F', 'NS(=O)(=O)/C=c1\\nc(Cl)nc(Nc2ccccc2)c1=O', 'Oc1ccc(O[C@H](F)c2cccc([N+](=O)[O-])c2)cc1F', 'Clc1cccc2cc(O)c(F)cc12', 'O=[N+]([O-])c1c(Br)scc1[C@@H]1Nc2ccc(Br)cc2N1', 'Clc1ccc(Br)cc1C1=c2ccccc2=[NH+]C1', 'Clc1cc(/N=C\\C=C/CC(=O)[O-])c(Br)s1', 'Clc1ccc(Br)cc1/N=N\\c1ccccc1Br', 'Clc1cc(N2CCc3nc(CSc4ncccc4O)[nH]c3N2)cs1', 'Clc1cccc(Br)c1Oc1ccc2c(c1)OCO2', 'Cc1sc(-c2cccs2)cc1[C@H](C)NC(=O)Nc1ccc(C#N)cc1', 'Clc1cc(F)c(C(F)(F)F)cc1OC1CCC1', 'NS(=O)(=O)/C=C/c1ccc2c(c1)OCO2', 'Oc1ccc(/N=c2/[nH]c3cc(Br)c([N+](=O)[O-])cc3s2)cc1O', 'Clc1cc(Nc2ccc(Cl)c(Cl)c2)c2ccccc2c1Cl', 'Clc1cc(F)cc(N2CC[C@@H]([NH2+][C@H]3CCNC3=O)C2)c1', 'O=[N+]([O-])c1c(Br)c(O)c2c(c1C(N)=O)CC2', 'O=[N+]([O-])c1c(Cl)nsc1Nc1cccc(Br)c1', 'O=[N+]([O-])c1ccc2c(c1)N(Cc1cc(C(F)(F)F)cs1)CC2', 'Nc1c(F)cccc1N/C=C(\\C#N)C(=O)Nc1ccc(Cl)cc1', 'Cc1sncc1[C@@H](C)Oc1cnc(Oc2cccc(Cl)c2)c(C#N)c1', 'NS(=O)(=O)/N=C\\c1ccccc1Br', 'Clc1cccc2c1N[C@@H](c1ccc([N+](=O)[O-])cc1)[C@@H]1CC=C[C@@H]21', 'Clc1ccc([C@@H](O)c2ccc3c(c2)CCO3)cc1Cl', 'Clc1ccc(-c2cc(F)c(/C=C/C(=O)[O-])o2)cc1', 'Clc1ccc(C(F)(F)F)cc1Oc1ccccc1Br', 'N[C@@H](C#N)c1ccccc1Cl', 'Clc1cccc(/N=N/[C@@H]2C=CCC2)c1', 'Clc1cccc2ncn(CCCC(=O)Nc3nncs3)c12', 'Clc1cccc(-c2n[n-]c(Sc3cnnnc3)n2)c1', 'Nc1c(F)c(N2CCCC2)c([N+](=O)[O-])cc1O', 'Clc1cccc(F)c1Nc1ccc([N+](=O)[O-])cc1[N+](=O)[O-]', 'Clc1ccc(-c2cc(F)ccc2O)cc1N', 'Clc1ccc(-c2cc(F)c(Cl)cc2N)cc1', 'Clc1cc(Nc2c(Cl)cc(Cl)c(Cl)c2Cl)ccc1F', 'Oc1ccc(F)c(Cl)c1Cl', 'Oc1ccc(F)c(Cl)c1Cl', 'Oc1ccc(F)c(Cl)c1Cl', 'O=[N+]([O-])c1c(Cl)nsc1-c1cccc(F)c1', 'Clc1ccc(Br)c(/C([O-])=N/N2CC[NH2+]CC2)c1', 'ClC[C@@H]1C[C@@H]1c1cc(F)cc(F)c1', 'Oc1ccc(F)cc1NC(=O)Cc1ccc(F)c(Cl)c1', 'Oc1cc(COc2ccc(Cl)cc2)c(Cl)cc1Cl', 'Clc1ccc([C@@H](Cl)C(=O)Nc2ccc(F)c(F)c2)cc1', 'N#C/C(=C/c1cccs1)C(=O)Cn1cnc([N+](=O)[O-])c1', 'O=C([O-])[C@H]1COc2cc(Cl)ccc21', 'ClC(c1cc(Br)ccc1F)c1ccc2c(c1)OCO2', 'NS(=O)(=O)/C(=N/Nc1ncccc1Cl)c1cc(F)ccc1O', 'O=C(Cc1ncccc1O[C@H]1CCSc2ccc(Cl)cc21)c1ccccc1', 'O=C(c1cc(Br)cc(Cl)c1)[C@@H]1CCOC1', 'NS(=O)(=O)/C(C#N)=C1\\C=CC(=O)N[C@@H]1c1ccccc1', 'O=C(CSCCOc1ccccc1-c1ccccc1)NNC(=O)c1cnc2ccccc2n1', 'Oc1ccc([N+](=O)c2ccccc2)cc1Br', 'NS(=O)(=O)/N=C/c1ccccc1O', 'O=C(CSCNc1ncccc1Cl)Nc1nc2ccc([N+](=O)[O-])cc2s1', 'ClCc1csc([N-]S(=O)(=O)c2cccc(F)c2)n1', 'NS(=O)(=O)/C=c1/sc2nc[nH]c2cc1NC(=O)c1ccccc1F', 'O=[N+]([O-])c1c(Br)cc(F)cc1C(=O)Nc1cc(C(F)(F)F)ccc1Cl', 'Clc1nnnn1/N=C/Sc1ccc(C(F)(F)F)cc1Cl', 'Clc1ccccc1Oc1cc(C(F)(F)F)ccc1F', 'Clc1cc(F)cc(Nc2cc(F)ccc2F)c1', 'Oc1ccc([N+](=O)C=S)cc1', 'Clc1cc(F)c(F)cc1[C@H](O)c1cccc(Br)c1', 'ClC(c1cc(F)ccc1Cl)c1ccc2c(c1)CCO2', 'ClC[C@@H]1Oc2ccc(F)cc2[C@@H]1C1CCCCCC1', 'O=C(/N=C/c1csc2ccccc12)N[C@@H](CC#N)c1cccc(Cl)c1', 'N#Cc1c(O)cc(N)cc1C(=O)Nc1cccc(OCO)c1', 'Oc1ccc(F)c(F)c1F', 'Oc1ccc(F)c(F)c1Br', 'Clc1cccc2cc([C@H](O)C(F)(F)F)cc(-c3ccccc3)c12', 'NS(=O)(=O)/C=C\\c1c(Cl)cccc1Cl', 'Oc1ccc(F)cc1CNc1cc(Br)ccc1F', 'Oc1cc(C(F)(F)F)cs1', 'Clc1cc(F)c(/C=N/NC(=O)c2ccco2)s1', 'Oc1ccc([N+](=O)c2ccccc2CO)cc1O', 'O=C([C@H]1c2cc(O)ccc2Oc2ccc(F)cc21)NC1CCCC1', 'Clc1ccsc1NSc1ncccn1', 'O=C(c1cc([N+](=O)[O-])ccc1[N+](=O)[O-])N1CCC=CC1', 'O=[N+]([O-])c1c(O)sc2c1CC[NH+](Cc1cccc(F)c1)C2', 'N#C[C@@H]1CCCN(C(=O)[C@H]2C[C@@H]2c2ccc(F)c(F)c2)CC1', 'Oc1ccc(F)c(ON/C=[N+]([O-])c2ccc(Cl)cc2)c1', 'Oc1cc(C(F)(F)F)ccc1/C=N/SC1=NC2=NC=NC2=N1', 'Clc1ccc(CO)c(Cl)c1', 'N#C/C(=C\\c1cccs1)C(=O)Nc1cc(Br)cc([N+](=O)[O-])c1', 'NS(=O)(=O)/C=C/c1cccnc1Cl', 'ClCc1cc(Cl)cc2c1OCOC2', 'Nc1ncccc1[S@@](=O)Cc1nc[nH]n1', 'O=C(CSCc1ccco1)N/N=C/c1ccc(Oc2cc([N+](=O)[O-])ccc2Cl)cc1', 'N#C[C@H]1NC(=O)C[C@@H](CCl)N1c1ccccc1Cl', 'Clc1cccc([C@@H](c2cccs2)COc2ccc([N+](=O)[O-])cc2)c1O', 'Oc1ccc(F)c(O)c1', 'Clc1ccccc1O/N=C1\\SC(=C(Cl)Cl)N1c1cccc(F)c1', 'Oc1ccc(O[C@H]2C=[NH+]C\\C=C2c2cc(Cl)ccc2O)cc1', 'Clc1ccc(Br)c(Cl)c1C(=O)Nc1ccc(C(F)(F)F)cc1', 'NS(=O)(=O)C[C@H]1Oc2ccccc2C(=O)N1c1ccc(F)cc1F', 'ClC[C@H]1CNc2ccc(Cl)cc21', 'O=[N+]([O-])c1c(Br)cccc1[O-]', 'C[C@@H](c1cccc(C#N)c1)N(C)C(=O)CNC(=O)[O-]', 'O=C(c1cc(Br)ccc1O)N1CC[C@@](O)(C(F)(F)F)C1', 'Oc1ccc(Oc2ccc(Cl)cc2)c(O)c1', 'Clc1cc(NSc2nncc3c(O)cccc23)no1', 'Clc1ccc(Br)cc1/N=N\\c1ccc(Br)cc1', 'O=C(c1c[nH]c([N+](=O)[O-])c1)c1ccc(Br)s1', 'Oc1ccc(Oc2ccc(Br)cc2)cc1O', 'O=C(c1ccsc1C#N)[C@@H]1Cc2ccccc2C(=O)N1c1ccc(F)cc1', 'Oc1ccc([C@H]2C[C@@H]2C(=O)[O-])cc1F', 'OC[C@@]1([C@H]O)NCc2ccc(Br)cc21', 'O=C(Cc1nc[nH]n1)Nc1c(Br)cc(Br)s1', 'Clc1ccc(-c2cc(F)c(O)cc2)c(F)c1', 'Clc1ccc(-c2cc(F)c(Cl)cc2O)cc1', 'O=C(CSCc1csc(NC(=O)c2ccc(Cl)cc2)n1)Nc1ccccc1', 'ClCc1cc(Br)ccc1O', 'Clc1ccc(Br)cc1O[C@H]1C=CCCC1', 'Oc1ccc(Oc2cc(F)cc(F)c2)cc1', 'Oc1ccc(Oc2cc(Br)ccc2F)cc1', 'Oc1cc(F)ccc1Oc1ccc(F)cc1', 'Cc1snnc1-c1cc2cc([N+](=O)[O-])ccc2o1', 'Clc1cc(C2=NN=C([S-])N2)ccc1OCO', 'Cc1snnc1Sc1nc(N)c2c(C)cccc2n1', 'NS(=O)(=O)/C(=N/O)c1cccc([N+](=O)[O-])c1', 'ClCc1cc(Br)c2c(c1)OCO2', 'O=[N+]([O-])c1c([N+](=O)[O-])c(Br)sc1[O-]', 'Clc1nncnc1[N-]S(=O)(=O)c1cc(F)cc(F)c1', 'Clc1ccc(Br)cc1/C=N/N1CC[NH2+]CC1', 'Oc1cc(Br)c(Br)cc1OC(F)F', 'ClC[C@@H]1CNc2ccc(F)cc21', 'Oc1cccc(Br)c1Oc1ccccc1C#N', 'Cc1noc(CCc2c(C#N)c([O-])nn2-c2cccc(C)c2)c1C', 'O=[N+]([O-])c1ccc(C(=O)N2N=C/C(=C/c3ccccc3Cl)C2=O)cc1', 'Clc1ccc(CO)c(N2C(=O)CNC2=O)c1', 'CNS(=O)(=O)c1ccc(Cl)c(C#N)c1Cl', 'Clc1ccc(-c2cccnc2S(=O)(=O)C(F)(F)F)cc1', 'Clc1cc(/C=N\\c2ccc(Cl)cc2)c(F)cc1[N+](=O)[O-]', 'Clc1cccc(Br)c1O', 'C=C([O-])c1ccc(N2CC(=O)NC3=C2CC[C@H]3C)cc1Cl', 'Oc1ccc([N+](=O)[O-])c(C2=NC=CCS2)c1[N+](=O)[O-]', 'N=Cc1cc2cc(OC)ccc2c(Cl)c1Cl', 'Oc1ccc([N+](=O)[O-])c(-c2ccc(F)cc2)c1', 'ClC([O-])=NS(=O)(=O)c1c(F)cc(Cl)cc1Cl', 'Clc1ccsc1/N=C/c1ccc(Sc2ncc[nH]2)o1', 'Clc1ccc(Br)cc1OS(=O)(=O)c1cccnc1Cl', 'Clc1ccc(Br)c(/C=C2\\N=C(c3ccccc3)N=N2)c1O', 'Clc1ccc(-c2cccnc2N2CCSCC2)cc1', 'Clc1ccc(-c2cccnc2N2CCCSC2)cc1', 'Oc1cc(F)c(Nc2ccc(Br)cc2)cc1Cl', 'Clc1ccc(C(=O)N2CCCNC2)cc1F', 'O=C(/N=C/C(=O)Nc1ccc(OC(F)(F)F)cc1)c1ccc([N+](=O)[O-])cc1', 'N#C[C@@H](Nc1ncccc1O)c1cccc(-n2cccn2)c1', 'Oc1cccc(Oc2ccccc2C(F)(F)F)c1', 'Clc1ccsc1Sc1ncccc1C(=O)Nc1ccc(F)c(Cl)c1', 'Oc1ccc([N+](=O)c2ccc(Br)o2)cc1', 'Clc1cc(Cl)c2c(c1)OCO2', 'Oc1ccc([N+](=O)[O-])c(/[NH+]=C/c2ccc(Cl)s2)c1', 'Clc1ccc([C@@H](Br)C(F)(F)F)c(F)c1', 'Clc1ccc(Br)cc1C(F)(F)C(F)F', 'Clc1cc(/C=C/[N+](=O)[O-])cc(C(=O)[O-])c1', 'Nc1[nH]nc2c1/C(=C(/[O-])Nc1ccc([N+](=O)[O-])cc1)S2', 'ClCc1csc2nc(SCc3cccnc3)nnc12', 'Clc1cccc([S@@](=O)Cc2ccc(-c3ccco3)s2)c1', 'O=C(CSCNS(=O)(=O)c1cc(F)ccc1F)Nc1ccccc1F', 'Clc1ccc(Br)cc1/N=[NH+]/N=C1\\C(=O)C=C(c2cccnc2)S1', 'Oc1ccc([N+](=O)[O-])c(Cl)c1Cl', 'CNC(=O)N[C@@H](C)c1ccc(F)cc1F', 'Clc1ccc(C(N)=[NH2+])cc1F', 'Clc1ccc(Br)cc1C(=O)[O-]', 'Clc1ccsc1/N=N/c1ccc(Cl)cc1Br', 'Clc1cc(Cl)c(Br)c(Cl)c1Cl', 'Cc1sc2nc(/C=C/c3cc(N)c(C#N)cn3)[nH]c(=O)c2c1C', 'Oc1ccc([N+](=O)C2CC2)cc1', 'Clc1ccc(C(N)=Nc2ccc([N+](=O)[O-])cc2)cc1', 'CN=C(/C#N)c1cccc(C#N)c1', 'Clc1ccc(Br)c(/N=[N+](\\[O-])c2ccc([N+](=O)[O-])cc2F)c1', 'Clc1ccccc1Oc1cc(SC(F)F)nc2cccnc12', 'N#Cc1csc(NCc2cnc3ccccn23)c1', 'Oc1ccc(F)c(C(=O)N2CCOCC2)c1', 'O[C@@H]1c2cc(C(F)(F)F)ccc2N[C@@H]2C[C@H]1C[C@H]2O', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(C(F)(F)F)c1C#Cc1ccccc1F', 'ClC(=S)Nc1ccc(Sc2ncccc2Cl)cc1', 'Oc1ccc([N+](=O)[O-])c(N[C@@H]2CC(=O)c3ccc(Cl)cc32)c1', 'Clc1cc(F)c(/N=C\\c2ccc(Cl)cc2C)cc1O', 'Oc1ccc([N+](=O)[O-])c(I)c1O', 'Oc1cccc(Br)c1C1CC[NH+](Cc2ccc(C#N)cc2F)CC1', 'N#Cc1cccc(NC(=O)COc2ccc(Br)cc2)c1', 'Clc1cc(NSc2nncs2)ccn1', 'O=C(CNCc1ccc([N+](=O)[O-])cc1)Nc1c(F)cccc1F', 'Cc1cc(Br)ccc1NC(=O)c1ccc(F)cc1Br', 'Oc1ccc([N+](=O)c2ccc(C(=O)[O-])cc2F)cc1', 'Oc1cc(F)c([C@H](O)C(F)(F)F)cc1', 'O=C(CC1=CCCCC1)NNC(=O)[C@@H]1CC(=O)N(c2ccc(C#N)cc2)C1', 'O=C(/C=C/O)C(=O)c1ccc(Cl)c(F)c1', 'O=C([O-])c1nnsc1Sc1ccccn1', 'Clc1cc(F)c([C@H]2CC(=O)N2c2ccc(Cl)cc2Cl)cc1', 'Clc1cc(Br)c(C(F)(F)F)cc1OC(=O)C1CC1', 'Cc1sc(C)c([C@H](C)NC(=O)c2ccc(C#N)cc2)c1C', 'Clc1ccc(C(=O)c2ccc(O)cc2F)cc1', 'Clc1cc(NC2=NC=CC2)ccc1O', 'Nc1c(F)c(F)c(F)c(F)c1F', 'Clc1cc(Cl)ncc1Sc1ncccc1C(=O)[O-]', 'Oc1ccc(F)c(Cl)c1', 'Clc1ccc(Br)c(O)c1', 'Oc1ccc([N+](=O)c2occc2Cl)cc1O', 'Clc1ccc(C(F)(F)F)c(F)c1', 'Clc1ccc(Br)cc1C(F)(F)F', 'Clc1cc(F)cc(Cl)c1C(F)(F)F', 'O[C@@H]1c2cc(Br)ccc2C[C@H]1Cl', 'Clc1cc(Cl)c(/N=C/c2cc(Cl)ccc2Cl)c(Br)c1O', 'Clc1cc(/N=C\\c2ccc([N+](=O)[O-])cc2F)cs1', 'O[C@@H]1C(c2ccc([N+](=O)[O-])cc2)=C(O)c2ccccc21', 'Oc1cc(-n2nnnc2Sc2ccccc2Cl)cs1', 'O=[N+]([O-])c1cc(Cl)ccc1Cl', 'Clc1ccc(C(=O)Oc2ccc(F)c(F)c2)cc1', 'Clc1cc(CO)ccc1C(F)(F)F', 'O=C(c1cc(Cl)cc2c1OCCO2)c1ccccc1', 'O=[N+]([O-])c1ccc2ccc(Br)cc2c1', 'ClCc1cc([N+](=O)[O-])ccc1I', 'Clc1ccc(C(F)(F)F)cc1F', 'C=C1Nc2ccc(C#N)cc2[C@H]1c1cccc(Br)c1OC(F)F', 'O=[N+]([O-])c1cc(Br)c(O)c(F)c1C(F)(F)F', 'O=[N+]([O-])c1cc(Br)c(Br)c(C(F)(F)F)c1O', 'Oc1cc(F)ccc1Br', 'Oc1ccc(F)cc1NC(=O)c1ccc([N+](=O)[O-])cc1O', 'Oc1ccc(F)cc1C(=O)Nc1ccc([N+](=O)[O-])cc1', 'Oc1cc(Br)cc2c1[C@H](N1CC[NH2+]CC1)OCO2', 'Cc1cc(NC(=O)C(=O)N[C@@H](C)c2ccsc2)n(C)c1', 'Clc1ccc(Br)cc1CO/[NH+]=C(\\N)C#N', 'ClCc1cc([N+](=O)[O-])ccc1Br', 'Oc1ccc([N+](=O)c2cccc(C(F)(F)F)c2)cc1', 'ClC[C@H]1COc2ccc(F)cc21', 'ClC[C@@H]1COc2ccc(F)cc21', 'Nc1nc(SCc2cccnc2)cs1', 'Clc1ccc(C(F)(F)F)cc1OC(=O)c1ccc(Cl)c(Cl)c1', 'Oc1ccc([N+](=O)c2ccc(Br)cc2)cc1Cl', 'Oc1ccc([N+](=O)[O-])c(-c2ccc(F)cc2F)c1', 'Oc1ccc(F)cc1[N+](=O)[O-]', 'Oc1ccc(F)cc1NC(=O)c1cc(F)ccc1O', 'Nc1c(Cl)cc(/C=C/Cl)c(Cl)c1', 'COc1ncnc(Sc2nn3c(=O)c(C)c(C)nc3c(=O)c3nc(C)sc23)c1N', 'Clc1cc(F)cc(I)c1O', 'Oc1ccc([N+](=O)c2ccc(Cl)cc2Cl)cc1', 'Oc1ccc([N+](=O)[O-])c(C(F)(F)F)c1', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n', 'Clc1ccc(Br)cc1O\n\n', 'Clc1ccc(Br)cc1O\n', 'Clc1ccc(Br)cc1O', 'Clc1ccccc1-c1csc(Sc2cccnc2)n1', 'Oc1cc(Br)ccc1[C@H](O)c1cc(Cl)ccc1Cl', 'O=[N+]([O-])c1c(Br)c(Br)cc([O-])c1[N+](=O)[O-]', 'Oc1ccc([N+](=O)c2ccc(F)cc2F)cc1', 'Oc1cc(F)ccc1C(F)(F)C(F)(F)F', 'Oc1cc(Br)cc2cc(F)ccc12', 'Clc1cc(Br)c([N+](=O)[O-])cc1Br', 'Clc1ccc(-c2cc([N+](=O)[O-])ccc2I)cc1F', 'Oc1ccc([N+](=O)c2cc(Br)ccc2O)cc1', 'Nc1c(F)c(F)cc(F)c1F', 'Clc1ccc(-c2n[nH]c(SCn3ccnn2)n3)cc1', 'O=[N+]([O-])c1c(Br)cc(Cl)cc1C(=O)N1CCOCC1', 'O=[N+]([O-])c1cc(F)c(Br)cc1Cl', 'O=C([C@H](Cl)O)c1ccc(Cl)cc1Cl', 'O=[N+]([O-])c1c(Br)cc(Br)cc1F', 'O[C@@H]1c2cc([N+](=O)[O-])ccc2Nc2cc(O)ccc21', 'Clc1cc(F)c(O)c(F)c1F', 'Oc1cc(O)c(Cl)cc1/C=C/C(=O)[O-]', 'Clc1cc(Br)c(/C=C/C(=O)[O-])cc1O', 'Oc1ccc([N+](=O)[O-])cc1/C=[NH+]/C(Cl)=C/Cl', 'Clc1cccc(C2=NOc3cc4ccccc4oc3=[NH+]2)c1', 'Clc1cc(NC2(n3cccc3)SCCN2)c2ncccc2c1', 'Oc1ccc([N+](=O)c2cc([N+](=O)[O-])ccc2O)cc1O', 'Oc1ccc([N+](=O)[O-])cc1Cl', 'Oc1cc(O)c(/C=C/C(=O)[O-])cc1Br', 'ClC([O-])=N/N=C/c1ccc(O)cc1O', 'Oc1ccc(O[C@H]2C=[NH+]/C(=C/c3cccnc3)SC2=S)cc1', 'Clc1nnc(SCc2ccc(F)cc2F)s1', 'Oc1ccc(F)c([O-])c1-c1ccc([N+](=O)[O-])cc1Cl', 'O=[N+]([O-])c1c([N+](=O)[O-])c(F)cc(Br)c1[O-]', 'O=[N+]([O-])c1c(F)cc(C(=O)[O-])cc1F', 'Clc1cc(/C=N/O)c(I)c(C(F)(F)F)c1[N+](=O)[O-]', 'Oc1ccc([N+](=O)c2cc(Cl)ccc2Cl)cc1O', 'Oc1ccc([N+](=O)c2cc(Br)ccc2F)cc1O', 'Oc1ccc([N+](=O)c2cc(C#N)cc(Cl)c2)cc1O', 'Oc1ccc([N+](=O)c2cc(O)ccc2F)cc1O', 'Oc1ccc([N+](=O)c2cc(F)cc(F)c2)cc1O', 'Oc1ccc([N+](=O)c2cc(C(F)(F)F)ccc2Br)cc1', 'O=[N+]([O-])c1c(F)cc(NN)c(O)c1O', 'Oc1ccc([N+](=O)C=O)cc1Cl', 'Oc1ccc([N+](=O)C=O)cc1Br', 'Oc1cc(Br)cc(Cl)c1C(F)(F)F', 'Cc1sn(-c2ccc(F)cc2)c(=O)c1/C=C(/C#N)C(=O)[O-]', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(Cl)cc1Br', 'O[C@@H]1[C@@H]([NH2+]CC2CCCCC2)CSCCN1C[C@H](O)C(F)(F)F', 'O[C@@H]1[C@@H](OCC(=O)OC2CCCCC2)[C@@H]2CCCC[C@@H]21', 'O[C@@H]1[C@@H](C(=O)[O-])[C@@H]1CC[C@@H]1CCCO1', 'O[C@@H]1C[C@@]2(OC(=O)C[NH+]3CCCC[C@@H]23)C1', 'O[C@@H]1COCC[C@@H]1[NH2+][C@@H]1CCOC2(CCC2)C1', 'O[C@@H]1CCCCCC[C@H]1[NH2+]C1CCCC1', 'O[C@@H]([C@@H]1CCOC2(CCOCC2)C1)[C@@H]1C[C@@H]1C', 'O[C@@H]([C@@H]([NH3+])CO)C(F)F', 'O[C@@H]([C@@H](SCC(F)(F)F)C(F)(F)F)NC[C@H]1CCCO1', 'O[C@@H](S[C@@H]1CCOC2(CCOCC2)C1)C(F)(F)F', 'O[C@@H](C[C@@H]1CCOC2(CCCCCC2)C1)C(F)(F)F', 'O[C@@H](C(=O)NC[C@@H]1CCC[C@H](S(=O)(=O)N2CCCC2)C1)[C@H]1CCS(=O)(=O)C1', 'OC[C@H]1[C@H]2CCCC[C@H]2[C@H]1O', 'OC[C@H]1O[C@@H](CO)[C@H](O)[C@H](O)[C@@H]2CCCC[C@H]12', 'OC[C@H]1OCC[C@H]2CCCC[C@@H]2C1', 'OC[C@H](C1CC[NH2+]CC1)N1CCOCC1', 'OC[C@H](Br)CC12CC3CC(CC(C3)C1)C2', 'OC[C@@]1([C@H]2CCC[NH2+]C2)CCC[NH2+]C1', 'OC[C@@]1(O)CCSC1', 'OC[C@@]1(N[C@@H]2CCS(=O)(=O)C2)CCCC1', 'OC[C@@H]1[C@H]([NH2+]C2CC2)CCC[C@@H]1CC[NH+]1CCCCC1', 'OC[C@@H]1CC2=C(C[C@H]1O)C(=O)O2', 'OC[C@@H](CC1CCOCC1)NNC(=O)[C@@H]1CCCO1', 'OC[C@@H](Br)C1CCC(C(F)(F)F)CC1', 'OCSC[C@H]1CCCC[C@@H]1[NH3+]', 'OCSC[C@@H](CO)[C@@H]1C[NH2+]CCO1', 'OCSCCCN1CC[NH+](C2CCCCC2)CC1', 'OCS1(=O)CCCCC1', 'OCS/C=C/C1=NN2CCCC[C@@H]12', 'OC1CC[NH2+]CC1', 'OC1=[N+]2cccc2C(=O)OC1', 'OC1=CC(=O)CC1', 'OC1(C[NH2+][C@@H]2CCOC3(CCCCC3)C2)CCCCC1', 'O=[N+]([O-])c1ccnc2cccnc1NC2', 'O=[N+]([O-])c1c(O)cc(NC[C@@H]2CCC[C@@H]2O)nc2n1CCCCC2', 'O=[N+]([O-])c1c(Cl)nc(C2CCCCC2)ncn1C1CC1', 'O=C([O-])[C@H](Br)[C@@H]1CCCO1', 'O=C([O-])CSCC[C@@H]1CCCO1', 'O=C([O-])C(C1CCOCC1)C1CCCC1', 'O=C([O-])/C=C/C=C/C=C/C1=CCCCC1', 'O=C([O-])/C(=N/O)N1CCOCC1', 'O=C([C@H]1[C@H]2C=C[C@@H](C2)[C@H]1C(=O)[O-])C1CCCCC1', 'O=C([C@H]1CC[C@@H]2CCCC[C@H]2[NH2+]1)N1CCOCC1', 'O=C([C@H]1CCCC[C@@H]1C[NH3+])[NH+]1CCCC1', 'O=C([C@H]1CC(=O)N(C2CCCC2)C1)N1CCCC[C@H]1CO', 'O=C([C@H](SC1CCCCC1)N1CCCC1)OC1CCCCC1', 'O=C([C@H](Cl)C(F)(F)F)N1CCOCC1', 'O=C([C@H](Cl)Br)[C@H]1CN2CCC12', 'O=C([C@H](Cl)Br)C(=O)[O-]', 'O=C(C[NH+]1CCN(C(=O)CCC2C[C@@H]3CCCC[C@H]3C2)CC1)NC1CCCCC1', 'O=C(CSCC(N)=N/O)[C@H]1CCC(=O)O1', 'O=C(CSC(NCC1(C(=O)O)CCCC1)N1CCCC1)N1CCCC1', 'O=C(CSC(F)(F)F)N1CCCSCC1', 'O=C(CS(=O)(=O)[C@H]1CCCC[C@@H]1[NH2+]C1CC1)N1CCC(CC(N)=O)CC1', 'O=C(CS(=O)(=O)[C@@H](C(=O)[O-])C(F)F)N[C@@H]1CCCC[C@@H]1O', 'O=C(CS(=O)(=O)C[C@H]1CCCCO1)NN1CCOCC1', 'O=C(CS(=O)(=O)CC1CCC1)NCC#N', 'O=C(COCCC12CC3CC(CC(C3)C1)[C@H]2C([O-])Cl)N1CCCCCC1', 'O=C(CNC(=O)C[NH2+]C1CC1)N1CC[C@](O)(CO)C1', 'O=C(CN(CC[NH+]1CCCC1)C1CCCCC1)N1CCCC1', 'O=C(CC[NH+]1CCOCC1)N[N+](=O)[O-]', 'O=C(CCNC(=O)N1CCOCC1)N1C[C@@H]2CCCN[C@@H]2C1', 'O=C(CCCC[NH3+])C1CCCCC1', 'O=C(CC1CC1)N1CCC(CC(=O)N(CCO)C2CC2)CC1', 'O=C(CC1([NH3+])CCC1)N1CCCCC1', 'O=C(CC(F)(F)F)N[C@@H]1C=C[C@@H](C(=O)[O-])C1', 'O=C(C1CCN(C2CCCC2)CC1)N1CCC2(CCSCC2)CC1', 'O=C(C1CCCCC1)N1CCC(N2CCOCC2)CC1', 'O=C(C1CCCC1)N1CC[C@H]([N+]2=CCCC2)C1', 'O=C(C1CCC1)N1CCN(C(=O)C2CC2)CC1', 'O=C(C1CC=CC1)N1CCC(OC[C@H]2C=N[C@H]3CS(=O)(=O)C[C@H]32)CC1', 'O=C(/N=C\\NC1=CN=NC1)\\CC(=O)[O-]', 'O=C(/C=C\\C(Cl)(Cl)Cl)[C@H]1CCS(=O)(=O)C1', 'O=C(/C=C1CCOC[C@@H]1N1C[C@H]2CCCC[C@H]12)N1CCOCC1', 'N[C@@H]1[C@H](Cl)CCC[C@@H]([NH+]2C[C@@H](C)O[C@H](O)C2)C1', 'N[C@@H](O)[C@@H]1CCCC[C@H]1[NH3+]', 'N[C@@H](O)CCN1CCCCC1=O', 'N[C@@H](NCC1(C)CC1)C(=O)N1CCC(C(F)(F)F)CC1', 'N[C@@H](C[NH3+])C(=O)NC[C@@H]1CCCO1', 'N[C@@H](CCC(F)(F)F)C(=O)N1CC[NH+]2CCCC[C@@H]2C1', 'NS(=O)(=O)N[C@@H]1CCCN(CC(F)(F)F)C1', 'NS(=O)(=O)NC1CCC2(CC1)OCCO2', 'NS(=O)(=O)N1CCN([C@H]2CCOC3(CCCC3)C2)CC1', 'NS(=O)(=O)N(C1CCCCC1)[C@@H]1CCC(=O)O1', 'NS(=O)(=O)N(C)[C@@H]1CCSc1Cl', 'NS(=O)(=O)N(C)CC(=O)N[C@H]1CCCC[C@@H]1OC1CCCCC1', 'NS(=O)(=O)C[C@H](Br)C(=O)[O-]', 'NS(=O)(=O)C[C@@H](O)CN1CCN(C[C@H](O)C(F)(F)F)CC1', 'NS(=O)(=O)CCCC1CC[NH+](CCCC2=CCCCC2)CC1', 'NS(=O)(=O)/N=C\\C1=CCCC[NH2+]1', 'NS(=O)(=O)/N=C/C[C@@H]1CCC[NH2+]1', 'NS(=O)(=O)/N=C/C1CCCC1', 'NS(=O)(=O)/C=C/C1CC1', 'N=CC[C@@H](O)C[NH+]1C[C@H](C)C[C@@H](C)C1', 'N=CCN1CC(=O)NC(=S)N1C[C@H]1C=CCC1', 'N=C1/N[N+](=O)OC1', 'N#C[C@H]([C@@H]1O[C@@H](O)[C@H]1CC(N)=O)[C@@H]1CCCC[C@H]1C(=O)[O-]', 'N#C[C@@]1(SCC(F)(F)F)CSCCO1', 'N#C[C@@]1(NC(=O)CN2CCCC2)CC1', 'N#CCCC[NH+](CCCBr)CC(F)(F)F', 'Clc1cccc2ccc(Br)cc1CN2C[C@@H]1C[C@@H]2CC[C@@H]1CO2', 'Clc1cccc2[nH]nc(NCCC3=CCCOC3)n2n1', 'Clc1cccc2[nH]nc(N/N=C/Cn1)CCCC2=O', 'ClCc1csc2s1CCCC2', 'ClC[C@H]1O[C@H]([N+]2CC2)C1(Cl)CCO', 'ClC[C@H]1OC[C@H]2CCCC[C@@H]2C1', 'ClC[C@H]1C[NH+]2CCCC[C@H]1C2', 'ClC[C@H]1CNCC[C@H]1C[NH+]1CCCC1', 'ClC[C@H]1CN2CCC[C@@H]2CO1', 'ClC[C@H]1CC[C@@H]([NH+](CC(=O)[O-])CO)C1', 'ClC[C@H]1CCOC2(CCCCC2)C1', 'ClC[C@H](O)[C@H]1CN2CCC[C@@H]2CO1', 'ClC[C@H](O)C1CC[NH+](C2CC2)CC1', 'ClC[C@H](Br)[C@@]1(O)CCOC1', 'ClC[C@H](Br)NC(=O)CC[C@H]1CCCO1', 'ClC[C@@H]1OCC[NH+]1[C@H]1CCOC2(CCSCC2)C1', 'ClC[C@@H]1OCC[C@@H]1CN[C@H]1CCCC[C@@H]1CO', 'ClC[C@@H]1OCCC[C@@H]1[NH2+]C1CC1', 'ClC[C@@H]1OCC(CN2CCC[C@@H]2C(N)=O)C1', 'ClC[C@@H]1COC[C@@H]1C1CCC1', 'ClC[C@@H]1CN2CCCC[C@@H]2CO1', 'ClC[C@@H]1CC[NH2+]C1', 'ClC[C@@H]1CC[C@@H](N[C@@H]2CCCC[C@H]2O)[C@H]1CO', 'ClC[C@@H]1C(=O)CCC(C2CC2)C1', 'ClC(F)/C=C/C(=O)N[C@@]1(CO)CCC[C@H]1CC1CC1', 'ClC(F)(F)C1CCN(C(=O)[C@@H]2CCCOC2)CC1', 'ClC(F)(C[C@@H]1CCO[C@]2(CCSC2)C1)C(F)(F)F', 'ClC(=S)[C@@H]1CCCS1', 'ClC(=S)O/C=C1\\C(/C=N\\C(=O)[O-])C(=S)NC(=O)N1', 'C[C@@H](OC[C@@H]1CCCO1)C(=O)OCC(=O)N1CCN(C)CC1', 'C[C@@H](O)C[C@@](C)(O)CNC[C@@H]1CCC[NH+]1C', 'C[C@@H](O)CN(C)[C@H]1CC[C@@H]([NH3+])C1', 'C[C@@H](N[C@@H](C)C(=O)N(C)C)C(=O)C1=CN(C2CC2)C1', 'C[C@@H](NC(N)=[NH2+])[C@H]1CCOC2(CCCCC2)C1', 'C[C@@H](CO)C[NH2+]CCOC[C@H]1CN(C)CCO1', 'COC[C@](C)(O)CNC(=O)N(C)[C@@H](C)C(C)(C)C', 'COC[C@](C)(C#N)NC(=O)CC12CC3C[C@](C)(C1)C[C@@](C)(C2)C3', 'COC[C@H](O)CN(C)C(=O)[C@@H]1CCC(=O)NC1', 'COC[C@@H]1[C@H](CC2CCCCC2)NC(C)=C1C(C)=O', 'COC[C@@H]([NH2+]CC1CCCC1)C1CC1', 'COC[C@@H](NC(=O)OC(C)(C)C)C(=O)NN', 'COCCCNC(=O)NCC[NH+]1[C@@H](C)CCC[C@@H]1C', 'COC(=O)[C@H]1CCCC(C)(C)[C@@H]1[NH3+]', 'COC(=O)[C@@]1(F)CC[NH+](CC(=O)[O-])C1', 'CN[C@H]1C[C@@H](C)N=C(C)[C@H]1O[C@H]1C[C@H](C)[C@H](O)C1', 'CN[C@H](C[C@]1(O)CCSC1)C(=O)[O-]', 'CN[C@H](CO)C1(C)CC1', 'CNS(=O)(=O)[C@H]1CC[NH2+]C[C@@H]1C(C)C', 'CNC(=O)[C@H]1[NH2+]CC[C@H]1C(=O)[O-]', 'CNC(=O)[C@H]1CN[C@@H]2CCCN2C[C@@H]1C(C)C', 'CNC(=O)[C@H]1CCC[NH+]1CC(C)(C)[NH2+]C', 'CNC(=O)CCC(=O)NC[C@@H]1CCCC[C@@H]1C', 'CN=C(C#N)CCC(=O)N1C[C@H](C)OCC1(C)C', 'CC[S@](=O)[C@H]1CCC[C@@H](NC(=O)N(CCC(F)(F)F)C2CCCC2)C1', 'CC[S@](=O)[C@@H]1CCCC[C@@H]1NS(=O)(=O)N1CCOCC1', 'CC[C@@H]1CN(C(=O)NCCNC(C)C)CCO1', 'CC[C@@H]1C(=O)NC[C@@H](C(=O)OC)C1', 'CC[C@@H](C[C@H]1CCCC[NH+]1C)C(=O)[O-]', 'CC[C@@H](CCO)NC(=O)N1C[C@@H](C)CCC[C@@H](C)C1', 'CC[C@@H](C(=O)NCCC1=CCCCC1)C(C)(C)C[NH+](C)C', 'CCO[P@]1(=O)[C@@](C)(C(=O)OC)[C@@]1(O)C(F)(F)F', 'CCN[C@](CCC(F)(F)F)C(=O)OC', 'CCNS(=O)(=O)C[C@@H](C)C#N', 'CCN1S(=O)C[C@@]2(C(=O)N3CCCCC3)CC(=O)N=C2N1', 'CCC[C@H]1CCC[C@@H]1C(=O)NCCCC[NH+](C)C', 'CCCS(=O)(=O)N1CCN(C(=O)[C@H]2CCCC[C@H]2[NH3+])CC1', 'CCCC(CCC(=O)[O-])C(=O)N1CCC(C)CC1', 'CCC(=O)N1CCC2(CC1)OC(C(=O)N1CCC(C)CC1)CC2', 'CC1=NO[C@@H](C(=O)N2CCC[C@H]2C(=O)[O-])C1', 'CC1=CC[C@]2(C)CCC[C@@H](C)C[C@H]1[NH2+]CC2', 'CC12CCCC(CCCC2)CC1(Br)Br', 'C=C[C@@H]1[C@@H](C)CC[C@@]1(O)C[NH+]1CCCC1', 'C=C[C@@H]1N=C2O/N=N[C@H]2C(=O)[C@@H]1C', 'C=C[C@@H]1CC[C@](C#N)([C@@]2(O)CCC(C)(C)[C@@H]2O)[C@@H]1O', 'C=C[C@@H](C(=O)OC(C)(C)C)N1C(=O)C(=O)N(C)C1=O', 'C=CC[NH+]1CCC(CCC(=O)N2CCCCCC2)CC1', 'C=CC[NH+](CC(=O)[O-])CC1CCCC1', 'C=CCO[C@@H](C)C(=O)N[C@@H]1CC[C@@H]([NH3+])C1', 'C=CCC[NH+](CC(C)C)[C@@H](C)C(=O)[O-]', 'C=CCC(C)(C)C[C@](C)(O)CC', 'C=C([O-])/C=C(\\C)NC(C)(C)C', 'C=C(C=S)C(=O)NCC1(O)CCCCC1', 'C=C(C#N)[C@@H](C#N)C[NH+](C)C/C(C)C']
length=  10003
#########
sorted_smiles_score= ['O=C(/C(c1cccs1)N1CCN(c2ccccc2OCc2c[nH]c3ccccc23)CC1)c1ccc2ccccc2c1', 'Clc1cccc2c1NC(=O)N2[C@H](C(=O)N(Cc1cccc(F)c1)c1ccccc1)c1ccccc1', 'Oc1ccc(Oc2cc([C@@H]3OCCN(c4ccccc4OC(F)(F)F)n3Cc3ccccc3)ccc2F)cc1', 'Oc1ccccc1-c1c(C(=O)NCc2ccccn2)cc(-c2ccccc2)n1Cc1ccccc1', 'Oc1ccc(/C=C(\\Cn2c3ccccc3c3ccccc32)c2ccccc2Cl)cc1', 'O=C(/C(c1ccccc1Cl)N1CCN(c2ccccc2)CC1)c1cccc2c1ccc1ccccc21', 'O=C(/N=C/c1ccc2c(c1)N(CCc1ccccc1)c1ccccc1-2)c1ccc[nH]1', 'O=C(c1cn(-c2ccccc2)nc1-c1cc2ccccc2nc1N1CCOCC1)c1ccc2ccccc2c1', 'Clc1ccc([C@@H]([O-])CNc2ccc3ccc4ccn4c(=O)n(Cc4cccc(F)c4)n3c2)cc1', 'Oc1ccc([C@H](/N=C2\\Oc3ccccc3C2=O)c2ccccc2Cl)cc1OCc1ccccc1', 'Clc1ccc([C@@H](N[C@@H](c2ccccc2)c2ccccc2)c2ccccc2)cc1', 'Clc1nnnc(CN2C(=O)/C(=C/c3cccc4ccccc34)c3ccccc32)c1-c1ccccc1', 'O=C(c1cn2ccsc2c1)N1CCC(c2[nH+]ccn2CCC(c2ccccc2)c2ccccc2)CC1', 'O=C(CNc1nn(-c2ccccc2)c2ccccc12)Nc1cnn(Cc2ccccc2)c1', 'Clc1ccccc1C1=C(c2ccc(OCn3cccn3)cc2)Nc2ccccc21', 'O=[N+]([O-])c1c(-c2nnnn2-c2ccccc2)nc(-c2ccccc2)c2ccccc12', 'O=C(c1cc(Cl)ccc1N1CC[NH+](C2CC2)CC1)N(Cc1ccsc1)C[C@@H]1CCCO1', 'O=C(Cc1c(N[O-])c2ccccc2c(=O)n1C1CCCC1)N1CCN(Cc2ccc3c(c2)OCO3)CC1', 'Oc1cccc(-n2ccc(-c3cnc(-c4cc5ccccc5[nH]4)cc3)n2)c1', 'O=[N+]([O-])c1c(-c2nc3ccccc3o2)ccc2nccc(NCc3ccccc3)c12', 'O=C([O-])c1nn2c(-c3cccnc3)cc(-c3ccc(Cl)cc3)nc2c1-c1ccccc1F', 'O=C(c1cn2ccsc2c1Nc1ccc(Br)cc1F)N1CCO[C@@H](c2ccccc2)C1', 'Clc1ccc(-c2ccccc2N2[C@H](Cl)[C@H](c3ccccc3)C3=C2CCCC3)cc1', 'ClC(c1cc2ccccc2c2onc(-c3ccccn3)c12)[C@H]1NC(=O)N(Cc2ccccc2F)C1', 'O=C([C@H]1c2ccccc2C(=O)NC12CCCN(c1cccc(Cl)c1)C2=O)Nc1cc(Cl)ccc1Cl', 'Clc1ccc(Br)cc1NCc1ccc2c(c1)CCC(=O)N2Cc1cc(F)ccc1F', 'Clc1ccccc1Oc1cc(-c2c[nH]c3ccccc23)nc2cnccc12', 'Oc1ccc(Oc2ccc(C(=O)Nc3ccccc3)c(-c3ccccc3)n2)cc1', 'Oc1cc(-n2nnnc2-c2c(F)cccc2F)c2ccccc2c1OCc1c(F)ccc(Br)c1', 'Clc1ccc(-c2nn(-c3ccc(Br)cc3)c3ccccc23)cc1', 'Oc1ccccc1/N=N\\C(=N\\OCc1ccccc1)c1nc(-c2ccc(-c3ccccc3)cc2)no1', 'O=C(/C(c1cccs1)(c1ccccc1)c1ccc2c(c1)C=CC2)c1ccccc1', 'Oc1ccc(Oc2cnc(Oc3ccccc3)nc2-c2ccccc2)cc1', 'Clc1ccccc1[C@H]1N/C(=C/c2ccc(-c3ccco3)cc2)N1c1ccc(F)cc1', 'ClC[C@@H]1OCCN(C(=O)Cc2ccccc2)C1', 'O=C(/N=c1\\[nH]c2ncnc(-c3ccccc3)c2c2ccc(O)cc12)c1cncs1', 'O=C(c1c[nH]nc1-c1cccc2c1cnn2-c1ccccc1)Nc1cc(F)ccc1F', 'Clc1cccc2cc(CN(Cc3ccnc4ccccc34)Nc3ccc(Br)cc3)sc12', 'O=C(/C=C/c1c(-c2ccccc2)[n+](-c2ccccc2)c([O-])[nH]c1=O)Nc1cccc(Cl)c1', 'O=C(c1cccc(NC(=O)N2CCN(c3ccc(F)cc3)CC2)c1)N1CCN(c2ccc(Cl)cc2)CC1', 'O=C([C@H](CN1CCOCC1)N1CCN(c2cccc(Cl)c2)CC1)C[NH+](CC1CC1)C1CC1', 'O=C(/N=c1/[nH]ccn1C1CC1)Nc1ccccc1N1CCN(c2ccccc2F)CC1', 'Clc1ccc(-c2ccccc2)cc1Nc1ccc2nnc(-c3ccccc3)n2c1', 'Clc1ccccc1Nc1c2oc3ccccc3c2c1C[NH+]1CCN(C(=O)c2ccccc2)CC1', 'Clc1ccc(Br)cc1-c1cn(-c2ccccc2)nc1-c1cccc2nccnc12', 'O=C(c1c[nH]c2ccccc12)N1CCC[C@H]1CN1CCc2ccccc21', 'Oc1ccc(/N=c2[nH]c3ccccc3n2Cc2ccccc2)cc1OCC1CCN(CO)CC1', 'O=[N+]([O-])c1ccc2cc[nH]c2c1CN1CCN(c2ccccc2)[C@@H]1c1ccc(-c2ccc(F)cc2)cn1', 'Clc1ccccc1Nc1cn(-c2ccccc2)nc1-c1ccccc1', 'Oc1cccc(-n2c(CCNc3ncnc4ccccc34)nc3ccccc32)c1', 'Oc1cccc(N2CCN(C(=O)c3ccc(C[NH+]4CCC(C(=O)Nc5cccc(Cl)c5)cc4)cc3Cl)CC2)c1', 'O=C(/C(c1cccs1)[NH+]1CCN(C(=O)c2cnn(-c3ccccc3)c2-c2ccccc2)CC1)c1ccc(C(F)(F)F)cc1', 'O=C(/C=C1/c(=C/C=C/C2=CCCCC2)[C@H]1c1cccc(Cl)c1)c1ccc2c(c1)OCO2', 'Clc1ccc(Br)cc1-c1cc(F)ccc1Cc1ccc(Br)cc1', 'O=C(CCOn1c2ccccc2c2ccccc21)Nc1ccc2c(c1)CCCC2', 'O=C(c1cn2ccsc2c1)N1CCn2nc(-c3ccc(F)cc3)cc2[C@@H]1c1ccccc1', 'Clc1ccc(-c2cc(-c3ccccc3O)nnc2-c2ccccc2)cc1', 'Clc1ccccc1Oc1cc(F)c(OCc2nc(-c3ccc(Cl)cc3)[nH]n2)cc1', 'O=C(Cc1c[nH]c2ccccc12)N1CCN(C(=O)c2ccccc2)CC1', 'Clc1ccccc1-c1nc(-c2ccccc2)sc1C(=O)N/N=C/c1ccccc1', 'O=C(CC[C@@H]1c2ccccc2C(=O)N1Cc1cccc(C(F)(F)F)c1)N1CCN(c2cccc[nH+]2)CC1', 'Clc1ccccc1[C@H](Cl)c1ccccc1OCc1ccccc1Cl', 'Clc1cccc([C@@H]2CC[C@@H]2CN2CCO[C@@H](c3ccccc3)C2)c1', 'O=C(COc1cc(Cl)c2[nH]c3c(c2c1)CCCC3)NCC1CC[NH+](Cc2ccccc2)CC1', 'Oc1ccccc1/C(=N\\Nc1cccc(Cl)c1[N+](=O)[O-])C1(c2ccc(Br)cc2)CCCC1', 'O=C(COc1ccc2ncccc2c1)Nc1ccc2nc(-c3ccc(Cl)s3)[nH]c(=O)c2c1', 'Clc1ccc(C(=O)NCCCN2CCO[C@@H](c3ccccc3)C2)cc1', 'O=C(COc1cc(N2CCCCC2)ncc1O)N1CCN(C(=O)c2cc(F)ccc2Br)CC1', 'Clc1ccc(-c2cc(Nc3ccccc3Cl)nc(-c3ccc(Cl)cc3)n2)cc1', 'N=C1/C(=C/c2c(OCc3ccccc3)sc3ccccc23)C(=O)N1c1ccccc1', 'Clc1ccccc1Oc1cc(Oc2ccccc2)nnc1-c1ccccc1', 'Clc1ccccc1OCc1ccc(-c2cccc3ccccc23)o1', 'O=C(c1cn[nH]c1N1CCN(c2ccccc2C(=O)N2CCCC2)CC1)N1CCC[C@H]1c1ccccc1', 'Clc1ccccc1Oc1cc(OCc2ccccc2)nc(-c2ccccc2)n1', 'O=C(Cc1coc2ccccc12)Nc1ccccc1-c1ccccc1', 'Oc1ccc(Oc2cncc(-c3ccccc3COc3ccccc3)c2)cc1', 'Clc1ccc(-c2nnnn2[C@@H](c2ccccc2)c2ccccc2F)cc1', 'Clc1ccccc1-c1cscc1-n1ccc2ccccc21', 'Clc1ccccc1Nc1nnnn1-c1cccc2ccccc12', 'Clc1ccccc1Cn1cnc2ccccc21', 'Oc1ccc(Oc2nc(-c3ccccc3)oc2-c2ccccc2)cc1Cl', 'O=C(CCc1nnc2ccccn12)Nc1c2[nH]c(=O)c3ccccc3c2c(Cl)cc1Cl', 'O=C(c1cccn2cc(-c3ccccc3)nc12)Nc1ccc(Cl)cc1', 'O=C(CC(c1ccccc1)N(C1CC1)c1ccccc1)N1CC[C@@]2(CCC(=O)Nc3ccccc32)C1', 'Clc1ccccc1Oc1cnc(Cc2ccccc2)n(-c2ccccc2)c1=O', 'Clc1ccccc1[C@H](N[C@H](c1ccccc1)C1CC1)C1CCCC1', 'Clc1ccccc1OC1(Cl)CCN(C(=O)c2ccccc2I)CC1', 'O=C(CNc1c(-c2ccc(Cl)cc2)cnn1-c1ccccc1)NNC(=O)c1ccco1', 'O=C(/C(c1cccs1)[C@@H](c1ccco1)N1C2=C(CCCC2=O)c2ccccc21)N1CCNC1=O', 'Clc1ccccc1NC1CCN(c2ccccc2OCc2ccc(F)cc2)CC1', 'Clc1ccccc1[C@H]1[C@@H]([NH2+]Cc2ccc3c(c2)OCO3)CCCC1(CCO)C[C@@H]1OCCO1', 'Clc1ccc(-c2ccccc2N2CCO[C@@H](CO)C2)cc1', 'Clc1ccc(-c2csc3[nH]cc(-c4noc5ccccc45)c23)cc1', 'Clc1ccccc1[C@H](Cl)[C@@H]1CN(C2CC2)CCO1', 'Clc1ccc(C[NH+](Cc2ccccc2)CN2CCN(c3ccccc3C(F)(F)F)CC2)cc1', 'Clc1ccc(COc2nc3ccccc3c3sccc23)c2ccccc12', 'Clc1ccccc1CNC1CC1', 'Clc1ccccc1[C@H]1[C@@H](Cc2ccccc2)c(=O)n(-c2ccccc2)N1c1ccc2c(c1)OCO2', 'Oc1ccc([C@H](/N=C2\\Oc3ccccc3O2)c2c[nH]nc2-c2cc(F)ccc2F)cc1', 'Clc1cc(Br)c(CN2CCO[C@@H](c3ccccc3)C2)cc1Br', 'Clc1ccccc1C(=O)NCC(=O)N1CCO[C@@H](c2ccc(F)cc2F)C1', 'O=C(c1cn(-c2ccccc2)nc1-c1ccccc1)N1CCN(C2CC2)CC1', 'Clc1ccccc1CCCC(=O)Nc1ccccc1OCc1ccc(F)cc1', 'Clc1cccc2ccccc12', 'O=C(C[NH+]1CCN(c2ccccc2)CC1)N1CCN(Cc2ccc(F)cc2)CC1', 'Clc1cc(Br)c2nc(OCC(=O)N3CCN(C(=O)Cn4ccc5ccccc54)CC3)ncc2c1', 'O=[N+]([O-])c1c(Cl)nnc2cc(-c3ccccc3)n(-c3ccc(F)cc3)c12', 'Clc1ccccc1-c1n[nH]cc1COc1ccc(-c2ccccc2)nc1', 'Clc1ccccc1CC[C@H]1CN(C[C@@H]2CCCCO2)CCO1', 'O=C(c1c[nH]c2ccccc12)N1CCN(CC2CC[NH+](Cc3ccccc3)CC2)C1', 'Clc1ccc(C(=O)c2ccccc2)cc1', 'Clc1ccccc1Nc1cccc(Cn2ccc3ccccc32)c1', 'Clc1ccccc1[C@H]1[C@@H]2O[C@@H](COC(=O)c3ccccc3)[C@H]2C(=O)N1c1ccc(Cl)s1', 'O=C([C@H](c1ccc(F)cc1)N1CCN(c2ccc(Cl)cc2)CC1)c1ccc2ccccc2c1', 'Clc1ccc(-c2ccccc2CCn2cnc3ccccc32)cc1', 'Clc1ccccc1OCC(=O)N1CCC[C@@H](CCc2ccccc2)C1', 'O=C(c1c[nH]c2ccccc12)N(C[C@@H](O)c1ccc(Br)cc1)C1CCC1', 'Clc1ccccc1OCCN(C(=O)Nc1cccc2[nH]ccc12)C1CC1', 'Oc1ccccc1[C@@H]1Cc2c([nH]c3ccccc32)c2ccccc21', 'Clc1ccccc1OCCOc1ccccc1Br', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(Br)c1N1c2ccccc2CC[C@@H]1Cc1c[nH]c2ccccc12', 'O=C(COc1ccccc1)Nc1ccccc1N1CCCCC1=O', 'O=C(/C=C/c1ccc(C(F)(F)F)cc1)OCCCCc1ccccc1', 'Clc1ccc(CCCN2CCc3[nH]c4ccccc4c3C2)cc1', 'Clc1ccccc1Oc1cc(-c2ccc(O)cc2)nc2ccccc21', 'O=C(CNC(=O)CCc1c[nH]c2ccccc12)c1ccccc1', 'Clc1ccc(-c2nnnn2-c2cc(Nc3ccccc3)ccn2)cc1', 'Clc1ccc(-c2cccnc2)cc1', 'Oc1ccccc1-c1nnn(CCc2ccc(F)cc2)c1-c1ccccc1', 'Clc1ccc(-c2ccccc2Cl)cc1', 'Clc1ccc(-c2ccccc2Cl)cc1', 'Clc1ccc(-c2nn(-c3ccc(OCc4ccccc4)cc3)c(Cl)c2)cc1', 'Clc1cc(Br)cnc1-c1ccc(F)cc1OCC(=O)N1CCO[C@H](CCC(F)(F)F)C1', 'O=C(c1ccc(OCc2ccccc2)cc1)N1CCc2ccccc21', 'Clc1ccccc1NCC(=O)N1CCCCC1', 'Oc1ccccc1COC1CCN(C(=O)Nc2ccccc2Oc2ccc(F)cc2)CC1', 'Clc1cccc(/N=C/C2=CC(=O)CCC2=O)c1CN1CCC[C@@H]1CN1CCc2ccccc21', 'Clc1ccccc1O/C=C(c1ccccc1)N1CCN(c2ccccc2)CC1', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n\n', 'Clc1ccc(Br)cc1\n\n\n', 'Clc1ccc(Br)cc1\n\n', 'Clc1ccc(Br)cc1\n', 'Clc1ccc(Br)cc1', 'O=C(CNc1cnccn1)N(c1ccc2ccccc2n1)c1ccccc1', 'O=C(c1ccc(-n2cncn2)cc1)N1CCN(c2cc(-c3ccccc3)ncn2)CC1', 'Clc1ccc(C(=O)NCCOc2ccccc2)cc1', 'Clc1cccc2[nH]nc(-c3ncc(-c4ccccc4)nc3)c12', 'Oc1ccc(Oc2ccc(NCC(=O)N3CCCc4ccccc43)cc2)cc1', 'Clc1ccccc1OCc1ccc(Cl)cc1', 'Clc1ccccc1Oc1cc(-c2ccccc2)nc2ccc(F)cc12', 'Clc1ccccc1CO[C@H](c1ccccc1)c1ccccc1', 'Clc1ccccc1C1CC1', 'Clc1ccc(-c2ccccc2NC2CCCCC2)cc1', 'O=[N+]([O-])c1cc(Br)c(C(=O)N2CCN(c3ccccc3[N+](=O)[O-])CC2)cc1OC(F)F', 'Clc1ccccc1OCC1CCCC1', 'Clc1ccccc1Oc1cnc(-c2n[nH]c3ccccc23)cc1', 'Clc1ccc(C2CC2)cc1', 'Clc1ccc(C(C)C)cc1CN/C(=C/c1ccccc1)c1ccccc1', 'O=[N+]([O-])c1c(OCOc2ccc(Cl)cc2)ncnc1N[C@H](c1ccccc1)c1ccc(Cl)cc1', 'Clc1ccccc1OCCC1CCCCC1', 'Clc1ccccc1OCc1cccc(Oc2ccc(-n3cccn3)cc2)c1', 'Clc1ccccc1Oc1ccccc1F', 'O=C(C1CCN(c2ccccc2)CC1)c1ccc(-n2cccc2)cc1', 'O=C(/C(c1cccs1)c1ccccc1)[C@@H](c1ccccc1O)c1ccccc1', 'O=C(C1CC[NH+](Cc2ccccc2F)CC1)N1CCc2ccccc2[C@H]1c1ccccc1', 'Oc1cccc2ccccc12', 'Oc1cccc2ccccc12', 'Oc1cccc2ccccc12', 'O=C(Cc1cnc(-c2ccncc2)c(Cl)c1)NC(c1ccccc1)c1ccccc1', 'Clc1ccc(-c2ccccc2CCNC(=O)COc2ccccc2Cl)cc1', 'O=C([C@H](c1ccccc1)N1CCC(OC2CCCCC2)C1)N1CCN(c2ccccc2)CC1', 'O=C(c1ccsc1C1CC1)N1CCN(c2cnn(-c3ccccc3)c(=O)c2NCc2ccccc2F)CC1', 'Clc1cc(Cl)c(CCCOc2ccccc2Cl)c(Br)c1Cl', 'O=C(COCc1ccccc1)N1CCC(c2ccccc2)CC1', 'Clc1ccccc1CNC1CCN(CN2CCc3ccccc32)CC1', 'Clc1ccccc1COC1CCCCC1', 'O=C(CN(CCc1ccccc1)C1CC1)Nc1ccccc1Cl', 'O=[N+]([O-])c1c(-c2noc(-c3ccc(F)cc3)n2)nnn1-c1ccccc1', 'Clc1ccc(-c2cc(-n3cccn3)nc(-c3cc(F)ccc3F)n2)cc1', 'O=C(c1ccsc1COc1ccccc1)N1CCN(C(=O)COc2ccccc2)CC1', 'Clc1ccccc1OCCCOc1ccc(Cl)cc1', 'Clc1ccccc1OC1CCN(C(=O)c2c[nH]c3ccccc23)CC1', 'Clc1cccc(Nc2ccc(Cl)c(C(=O)Nc3nnc(-c4ccccc4)o3)n2)c1', 'Clc1ccccc1COc1ccccc1F', 'Clc1ccc(CN[C@@H](Cc2c[nH]c3ccccc23)C2CC2)cc1', 'Clc1ccccc1C(=O)OCCNC(=O)c1ccc(F)cc1', 'Clc1ccccc1OCCn1cccc(Br)c1=O', 'Clc1ccc(COc2ccc(Br)cc2)cc1', 'Clc1ccc(-c2cccs2)cc1', 'O=C(CNCc1c(F)cccc1F)N1CCN(Cc2ccc(F)cc2)CC1', 'O=C([C@H]1c2ccccc2C[C@@H]1Oc1ccc2c(c1)OCCO2)N1CCN(Cc2cn3ccccc3[nH+]2)CC1', 'O=C(/C=C1/N(Cc2ccccc2)c2ccccc2C1=O)Nc1ccccc1OC1CCCC1', 'O=C(c1cccn1-c1ccc(Br)cc1)N1CCC[C@H]1CC(=O)N1CCCC1', 'Clc1ccc(-c2nonc2Cn2cnc(-c3ccc(Cl)cc3)n2)cc1', 'Clc1cccc2[nH]nc(NCc3ccc(Oc4ccccc4)cc3)c12', 'Oc1ccccc1[C@H](NCc1nc(-c2ccccc2)n[nH]1)c1ccc(Cl)cc1', 'Clc1ccccc1CSCCc1ccccc1', 'Clc1ccccc1Oc1cnc(-n2cnc3ccccc32)nc1', 'Clc1ccc(Br)cc1-c1cn(-c2ccccc2)nc1-c1ccccc1', 'Clc1ccccc1NC1CCN(C(=O)c2ccc(OCc3ccccc3)s2)CC1', 'Oc1ccccc1-c1ccc(=O)n(Cc2nc(-c3cccc(F)c3)no2)c1', 'Clc1ccccc1[C@H](c1oc2ccccc2c1[N+](=O)[O-])N1CC[NH+](Cc2ccccc2)CC1', 'Clc1ccc(C(N)=O)cc1', 'Clc1ccc(C(N)=O)cc1', 'Clc1ccc(Br)cc1-c1cn(-c2ccccc2F)nc1-c1ccccc1', 'Oc1ccccc1-c1ccc(Cl)cc1', 'Clc1cc(Cl)ccc1Cl', 'Oc1ccccc1COc1ccccc1C(=O)NC[C@H]1CCCc2ccccc21', 'O=C(CN(C[C@H]1CCCO1)c1ccccc1)N[C@H](c1ccccc1)c1ccccc1', 'Oc1ccc(/C=C(\\Nc2ccccc2)C(=O)Nc2ccccc2)cc1', 'Clc1ccccc1Oc1cnc2ncnc(-c3ccccc3)c2c1', 'O=C([C@H](CNC(=O)c1ccc(N2CCOCC2)cc1)c1ccc(F)cc1)NNC(=O)c1ccoc1', 'Clc1ccccc1OCc1ccncc1', 'Clc1ccc(Br)c(OCCSc2ccc(Cl)cc2)c1', 'O=C(Cc1nn(Cc2ccccc2)cc1COc1ccc(Br)cc1)c1ccccc1', 'Clc1cc(Br)c(CCNC(=O)[C@@H](Cc2ccccc2)[NH+]2CCN(c3ccccc3F)CC2)nc1C1CC1', 'O=C(C1CCC(Nc2ccccc2Cl)CC1)c1ccc(Cl)cc1', 'Clc1ccccc1Nc1ccccc1Cl', 'Clc1ccccc1OCCC(=O)N1CCC[C@H](C(=O)N2CCCC2)C1', 'Clc1ccccc1CN(Cc1ccccc1)Cc1ccccn1', 'Clc1ccsc1SC[C@@H]1CN(Cc2ccc(Cl)cc2)CCO1', 'Clc1ccc(-c2csc3s/cnc23)n2c1nc1ccccc12', 'O=C(/N=c1\\sc2ccccc2c(=O)n1-c1ccccc1)C1CCN(C(=O)c2ccc(Cl)s2)CC1', 'Clc1ccc(-c2ccccc2COC2CCN(c3nc4ccccc4o3)CC2)cc1', 'Clc1nc(-c2ccccc2F)nc2c1cnn2-c1ccc(Br)cc1', 'Clc1ccc(CNc2cccc(Cl)c2)cc1', 'O=C([C@H]1c2ccccc2CCN1C(=O)C1(c2ccccc2)CCC1)NCc1ccco1', 'Clc1ccccc1Cn1c(SCc2ccccc2)nc2ccccc21', 'Clc1ccccc1C1CCN(CC(=O)Nc2ccccc2F)CC1', 'N=CCNC(=O)C1CCN(C(=O)CNC(=O)c2ccc(F)cc2)CC1', 'Clc1ccccc1OCCCn1ccc2ccccc21', 'Clc1cccc(-c2cnc(-c3ccc4ccccc4c3)o2)c1', 'Oc1cccc(NCc2ccc(-c3ccccc3)cc2)c1', 'O=C(CN(CCO)c1ccccc1)Nc1ccc(Cl)cc1', 'O=C(COCc1ccccc1)N/N=C/c1ccc(OCc2ccccc2)cc1', 'O=C(c1cc([N+](=O)[O-])ccc1N1CCN(c2ccccc2)CC1)N1CCC(OC2CCOCC2)CC1', 'Clc1cccc(Br)c1COCc1ccc(Cl)cc1', 'ClC[C@H]1CN(Cc2cccc(O)c2)CCO1', 'O=C(CCCNC(=O)C1(c2ccc(Br)cc2)CC1)Nc1ccccc1', 'Clc1ccc(C(=O)N2CCN(Cc3ccc(Cl)cc3)CC2)cc1', 'Clc1ccccc1OCc1ccc(F)cc1NC(=O)c1ncn(Cc2ccccc2)n1', 'Clc1ccccc1OCc1cc(-c2ccccc2O)nc2ccccc12', 'Oc1cc(COCc2cccc(Br)c2)c2c(cc3ccccc32)c2ccccc12', 'Clc1cc(/C=C(\\Nc2ccccc2)c2ccccc2)ccc1Cl', 'O=C(c1cn(-c2ccccc2)nc1-c1cnn(-c2ccccc2)c1)C1CCOCC1', 'Clc1ccc(-c2nnnn2CCc2cnc3ccccc3n2)cc1', 'O=C(C1CC[NH+]([C@H]2CC[NH+](Cc3c[nH]c4ccccc34)CC2)C1)NCCOc1ccccc1', 'Oc1ccccc1-c1ccccc1F', 'Clc1ccc(Br)c(Cl)c1CCOCC1CCN(C(=O)[C@@H]2CC=CCC2)CC1', 'Clc1ccccc1NCc1ccccc1N1C(=O)CCc2ccccc21', 'Clc1ccccc1OCCSc1ccccc1F', 'Clc1ccc(CCOc2ccc3ccccc3c2)cc1', 'Clc1ccccc1Nc1c2ccc(Oc3ccccc3)nn2c(=O)cn1', 'Oc1ccccc1Cc1ccccc1OCc1ccccc1Cl', 'O=C(COc1cccc2ccccc12)Nc1ccc(N2CCOCC2)cc1', 'Clc1ccccc1[C@H](NCc1ccc(Br)cc1)c1ccccc1', 'Clc1ccccc1OCc1ccc(C(=O)NCc2ccco2)cc1', 'Oc1ccccc1-c1ccccc1COc1ccccc1Cl', 'Clc1ccccc1C1CCN(CCSc2ccccc2)CC1', 'Clc1ccccc1O/N=C\\c1ccc(COc2ccccc2)cc1', 'Clc1ccccc1OCc1ccc(N2CCOCC2)cc1', 'O=C(Cc1coc(-c2ccccc2)n1)NCc1ccccc1', 'Clc1cccc(/N=C/c2ccc(-c3ccccc3)cc2)c1', 'Clc1ccccc1NS(=O)(=O)c1ccc(Oc2ccccc2)cc1', 'CCn1c(=S)[nH]c2ccccc21', 'Clc1cc(CSC[C@@H](CSc2ccccc2)c2ccc(Br)cc2)c(F)c2c1CCCC2', 'Clc1ccc(-c2ccccc2)nn1', 'O=C(c1c[nH]c(NN=C(c2ccccc2)c2ccccc2)n1)NNc1ccc(F)cc1', 'Clc1ccccc1CCCCC(=O)Nc1ccc(Cl)cc1F', 'Clc1ccccc1COCCSc1ccccc1', 'O=C(CCNS(=O)(=O)c1ccccc1)Nc1ccccc1', 'Clc1ccc(-c2cccnc2Cl)cc1', 'Clc1ccc(-c2cccnc2Cl)cc1', 'Clc1ccc(-c2cccnc2Cl)cc1', 'Clc1ccc(Br)cc1-c1ccccc1', 'Clc1ccc(Br)cc1-c1ccccc1', 'O=C(C1=C([O-])C(=O)N(C[C@@H]2CCCO2)[C@@H]1c1cccc(Br)c1)c1ccc2c(c1)OCO2', 'Clc1ccccc1CNC1(c2ccccc2)CC1', 'Clc1ccc(Br)cc1OCCOCc1ccccc1', 'Clc1ccc(Cc2ccccc2)c(F)c1', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n\n', 'Clc1cccc2cccnc12\n\n\n', 'Clc1cccc2cccnc12\n\n', 'Clc1cccc2cccnc12\n', 'Clc1cccc2cccnc12', 'Clc1cccc2cccnc12', 'Clc1cccc2cccnc12', 'Clc1cccc2cccnc12', 'Clc1cccc2cccnc12', 'Clc1ccccc1Oc1cc(Br)c(COc2ccc(Br)cc2)nc1N1CCOCC1', 'Clc1cccc([S@@]Cc2cccc3ccccc23)c1', 'Clc1ccccc1Nc1ccnc2ccccc12', 'Oc1cc(O[C@@H]2Oc3ccccc3Oc3ccccc32)ccc1N1CCC[NH+]1CCc1ccccc1', 'Clc1ccc(-c2csc3ccccc23)cc1', 'O=[N+]([O-])c1c(Cl)nn(CC(=O)N2CCO[C@@H](Cn3ccc(Br)c3)C2)c1C1CC1', 'Clc1cccc(F)c1NCc1ccc(F)cc1', 'Clc1ccc(CN2CCN(C3CC3)CC2)cc1', 'O=C(CNCc1noc(Cc2ccccc2)n1)N1CCN(c2ccccc2F)CC1', 'CC1CCN(C(=O)c2cc(C(=O)N(Cc3ccccc3)c3ccccc3)ccc2F)CC1', 'Nc1[nH]nc2ncc(-c3ccc(-c4ccccc4)cc3)nc12', 'Clc1ccc(COCOc2ccccc2F)cc1', 'Clc1ccsc1[C@H](N[C@H]1CCOc2ccccc21)[C@H]1CCCC[C@@H]1C(=O)N1CCCC1', 'Clc1ccc(Br)c(F)c1-c1ccccc1OCc1cc(F)cc(Cl)c1', 'O=C(COc1cc(Cl)ccc1Cl)N1CCc2ccccc21', 'Clc1ccsc1/CCN(C(=O)c1ccc(N(C2CC2)c2ccccc2)cc1)C1CC1', 'Clc1ccc(Cc2noc(COc3ccccc3)n2)cc1', 'Clc1ccccc1OCc1ccc(NC[C@@H]2COc3ccccc3O2)cc1', 'Clc1ccc([C@@H](CCCCl)NC(=O)c2ccccc2)cc1', 'Clc1ccc(Br)cc1-n1cnc2ccccc21', 'Clc1ccc(Br)cc1-n1cnc2ccccc21', 'Clc1ccc(-c2nnnn2Cc2ccccc2)cc1', 'Clc1ccccc1CSCCOc1ccccc1F', 'CCc1cccc(Oc2ccccc2NC(=O)Cc2ccccc2)c1', 'Clc1cccc(C(=O)NC2CC2)c1', 'Clc1ccc(Br)cc1CCc1ccccc1', 'Clc1ccccc1NCc1ccc(Cl)c(Cl)c1', 'Clc1cccc(F)c1Oc1ccccc1', 'Oc1ccccc1C(=O)OCc1ccc(F)cc1', 'Clc1ccccc1OC1CCCC1', 'Oc1ccc(Oc2ccc(F)cc2)cc1CN1CCN(Cc2ccccc2)CC1', 'Clc1ccc(Br)cc1-n1ncc2ccccc21', 'O=C(COCc1ccccc1)Nc1cccc(OCc2ccccc2)c1', 'Clc1ccccc1OCCn1c(=O)[nH]c2ccccc21', 'O=C(c1c[nH]c2ccccc12)N(CCO)c1ccccc1', 'Clc1ccc(-c2cc(Cl)ccc2Cl)cc1', 'Clc1ccccc1NS(=O)(=O)c1ccc(Cl)cc1', 'O=C(c1cc(Br)cn1CC(F)(F)F)N1CCC[C@H](C(N2CCCCC2)c2ccccc2F)C1', 'O=C(/C=C/c1ccnc2c1-c1ccccc1O2)N1CCC(Cc2ccccc2)CC1', 'COc1ncnc(NNC(=O)c2cc3ccccc3nc2-c2ccccc2)n1', 'Oc1ccc(F)c(/N=C\\Nc2ccccc2F)c1NS(=O)(=O)c1ccccc1', 'Clc1ccccc1Cn1cnc2cc(F)ccc21', 'Clc1ccccc1NS(=O)(=O)c1ccc(F)cc1', 'Clc1ccccc1NS(=O)(=O)c1ccc(F)cc1', 'O=C(COCCNc1ccccc1N1CCOCC1)NCc1ccccc1', 'Clc1ccccc1CCCOc1cccc(Br)c1', 'Clc1ccc(-c2ccccc2)cc1O', 'Clc1ccccc1Oc1cc([N+](=O)Nc2ccccc2)ccc1N1CCN(C(=O)c2ccccc2)CC1', 'Clc1ccccc1Oc1cc(Cl)ccc1N1CCCCC1', 'Clc1cc(Nc2c(cnn2Cc2ccco2)c2ccccc2)ccc1[N+](=O)[O-]', 'Oc1ccc(/C=N/Nc2ncnc(Nc3ccccc3)c2N2CCCC2)c2c1CCCC2', 'Clc1ccccc1COCCSc1ccc(F)cc1', 'Oc1cccc2ncccc12', 'Oc1cccc2ncccc12', 'Oc1cccc2ncccc12', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n\n', 'Clc1ccc(Br)cc1Cl\n\n', 'Clc1ccc(Br)cc1Cl\n', 'Clc1ccc(Br)cc1Cl', 'Clc1ccc(Br)cc1Cl', 'Clc1cc(Br)ccc1Cl', 'C=CCn1cc(C(=O)NNC(=O)N2CCN(c3ccc(Cl)cc3)CC2)c2ccccc21', 'O=C(CCOc1cccc(Cl)c1)NCc1ccccc1Cl', 'Clc1ccc(-c2ccccc2C(F)(F)F)cc1', 'O=C(/C=C/c1ccc(Br)cc1)N(C[C@H]1CCCO1)c1ccccc1', 'Clc1cccc(F)c1CNCc1ccccc1F', 'Oc1cc(CCn2c(=O)[nH]c3ccccc32)c(Cl)cc1OCc1ccccc1', 'Clc1ccc(-c2nnnn2-c2ccccc2)cc1', 'Clc1ccccc1CN(c1ccccc1)S(=O)(=O)c1ccc(Cl)cc1', 'Clc1cccc2ccc(Cl)cc12', 'Clc1cccc2ccc(Cl)cc12', 'Clc1ccccc1COCc1cccc(COc2ccccc2)c1', 'O=C(c1ccsc1)Nc1ccc(C(=O)Nc2ccccc2)cc1', 'Oc1cccc(OCc2cccc3ccccc23)c1', 'Clc1ccc(-c2cc(F)ccc2F)cc1', 'Clc1ccc(-c2cc(F)ccc2F)cc1', 'Oc1ccccc1-c1cccc2cccnc12', 'Clc1ccccc1Oc1cc(NC(=S)Nc2ccccc2)ccc1Cl', 'Clc1ccccc1NC(=O)N1CCN(c2ccccc2OC(F)F)CC1', 'Oc1ccc(F)c(OCc2ccccc2)c1', 'Clc1ccccc1OCc1ccc(Cl)cc1Cl', 'Clc1cccc2c1-c1ccccc1C2', 'O=C(c1ccc(Cl)cc1)N1CCOCC1', 'Clc1ccccc1NC(=S)Nc1ccc(F)cc1', 'COc1nn[nH]c1CCC(=O)N1CCC[C@@H](C(=O)NCc2ccccc2F)C1', 'Clc1ccccc1OCc1csc2ccccc12', 'CNc1cccc(OCC(=O)N2CCCC2)c1', 'Clc1cccc2c(N)cccc12', 'O=C(COc1cc(Br)c(O)cc1)N1CCN(S(=O)(=O)c2ccc(Br)cc2)CC1', 'O=C(c1cccn1Cc1ccccc1)N1CCN(c2ccc[nH+]c2)CC1', 'Clc1cccc(NC2CCCC2)c1', 'Clc1ccccc1-c1cnccn1', 'Clc1ccccc1-c1cnccn1', 'Clc1ccccc1NS(=O)(=O)c1ccc(Br)cc1', 'Oc1ccc(F)cc1COc1ccccc1', 'O=C(c1cccc(Br)c1)Oc1ccc(Cl)cc1', 'Oc1cccc(-c2cc(CNc3ccc(Cl)cc3)nn2-c2ccccc2)c1', 'Clc1ccccc1COCCNS(=O)(=O)c1ccccc1', 'O=C(COc1ccc(F)cc1)NNC(=O)c1ccccc1Br', 'Clc1ccccc1CSc1ccc(CO)cc1', 'O=C(C1CC[NH+](CCC(=O)N2CCNC[C@H]2CO)C1)N1CCN(c2ccccc2[N+](=O)[O-])CC1', 'ClC(=S)c1ccc(N2CCC(COc3ccccc3)CC2)cc1', 'Clc1ccc(Br)c(Cl)c1COCc1ccccc1Br', 'Clc1cccc(Nc2cccc(COc3ccccc3)c2)c1', 'Oc1ccc(F)c(SCC(=O)N2CCO[C@@H](c3ccc(Cl)cc3)C2)c1', 'ClCc1cccc(OCCc2ccccc2)c1', 'Clc1cccc(Br)c1OCCc1ccccc1', 'Oc1ccc(Oc2ccc(C=O)cc2)cc1', 'Clc1ccc(COc2ccc(F)cc2)cc1F', 'CC(C)OCCc1n[nH]c(-c2ccccc2)n1', 'Clc1ccccc1-c1noc(Cc2ccccc2)n1', 'Clc1ccc([C@@H](N[C@@H]2CCc3ccccc32)c2ccccc2)cc1', 'O=C(C1CCN(c2ccc([N+](=O)[O-])c(F)c2)CC1)N1CCC[C@H]1CC(=O)Nc1ccccc1Br', 'Clc1ccc(Br)cc1SCc1ccc(F)cc1', 'Clc1ccc(Cc2cccc(CNC(=O)C3CC3)c2)cc1', 'Clc1ccccc1CN(Cc1ccsc1)Cc1ccc(F)cc1', 'Clc1cccc(Br)c1NC[C@@H]1CCCc2ccccc21', 'Clc1ccccc1OCc1ccc(Oc2ncnc(Nc3cccc(F)c3)c2)cc1', 'Clc1ccccc1C(F)(F)F', 'Clc1ccc(Br)cc1OCCN1CCN(c2ccccc2)CC1', 'Clc1ccccc1NCc1cc(F)ccc1Cl', 'O=C(CCS)c1ccccc1', 'O=C(c1ccsc1)Nc1nnc(C(=O)Nc2ccc(-c3ccccc3)cc2)s1', 'Oc1ccccc1Oc1ccccc1-c1nc(-c2ccccc2)nn1CC1CC1', 'Clc1ccc(-c2nnnn2CCC(=O)N2CCCC2)cc1', 'Clc1cc(Cl)cnc1-c1ccccc1', 'Clc1ccccc1OCc1cc(Cl)ccc1F', 'Clc1ccc(-c2nn(-c3ccccc3)cc2CN2CCN(CCc3cccc(F)c3)CC2)cc1', 'Clc1ccccc1C(=O)Oc1cccc(Cl)c1', 'Clc1ccc(-c2nnnn2[C@@H](c2cccs2)c2ccccc2)cc1', 'Oc1cccc(N2CCCC2)c1', 'Clc1ccc(CNS(=O)(=O)c2ccc([N+](=O)[O-])cc2)cc1', 'O[C@@H]1c2ccccc2CN1Cc1ccc(-c2ccccc2)cc1', 'Clc1ccc(-c2nn(-c3ccc(-c4cccs4)cc3)nn2)cc1', 'Clc1cccc2cc(F)ccc12', 'Clc1cccc2cc(F)ccc12', 'Clc1ccccc1OCc1cc(Br)ccc1Br', 'Oc1cc(-n2cccn2)ccc1NC1=NC=CN=C1c1ccccc1Cl', 'Clc1ccccc1-c1ncc(COc2ccccc2)[nH]1', 'Clc1cccc(C(F)(F)F)c1', 'Clc1ccccc1NC(=S)Nc1cccc(Cl)c1', 'O=C(CNc1ccccc1[N+](=O)[O-])c1ccccc1', 'O=C(C1CC1)N(CCO)Cc1ccccc1Cl', 'Clc1cc(Br)c(Br)c(CCNc2ncnc3c2cnn3Cc2ccccc2)c1', 'Clc1ccccc1OCc1cc(F)ccc1F', 'CCNc1cccc(NC(=O)NC2CN(C(=O)CCc3ccc(C)cc3)C2)c1', 'Clc1ccccc1Oc1cncc2ccccc12', 'Clc1ccccc1Oc1cncc2ccccc12', 'Clc1cc(CC(=O)N2CCN(CCn3cncn3)CC2)ccc1N1CCCC1', 'Clc1ccc(Br)cc1NCCc1c[nH]c2ccccc12', 'Clc1ccc(-c2nnnn2Cc2ccccc2Cl)cc1', 'Clc1cc(CN(Cc2ccccc2)Cc2ccsc2)ccc1Cl', 'Clc1ccccc1Cn1nnc(-c2ccc(Cl)cc2)n1', 'Clc1ccc(Cc2cccc([N+](=O)[O-])c2)cc1', 'Clc1ccccc1CSCc1cccc(Br)c1', 'Clc1ccccc1OCc1csc(-c2ccccc2)n1', 'Oc1cc(-n2cnc(Cc3ccccc3)n2)ccc1Oc1ccc(Br)cc1', 'Clc1ccccc1OCCNc1ncccn1', 'Clc1cccc(Br)c1NCCCOc1ccccc1F', 'Clc1cccc(Nc2cc(Cl)ccc2Cl)c1', 'Clc1ccc(Br)c(SCc2ccccc2)c1', 'O=C([C@H]1COCCN1c1c(F)cccc1F)N1CCN(C(=O)COc2ccccc2)CC1', 'O=C(Cc1noc2ccccc12)Nc1ccccc1', 'Clc1ccc(Br)cc1COc1ccc(Br)cc1', 'Clc1cccc(F)c1Nc1cccc2ccccc12', 'Clc1ccccc1COCC1CCN(C(=O)c2ccco2)CC1', 'Clc1ccccc1Oc1cnc2scc(-c3ccccc3)c2c1', 'Clc1ccccc1Nc1ccc(F)cc1F', 'Oc1cccc(Br)c1Cc1ccccc1', 'Oc1ccccc1Cc1ccccc1Br', 'Clc1ccccc1Oc1ccccc1CN(Cc1ccccn1)C1CC1', 'Clc1ccccc1Oc1cc(-c2n[nH]c(COc3ccccc3)n2)c[nH]1', 'Clc1cccc(F)c1CSc1cccc(Cl)c1', 'O=[N+]([O-])c1cc(OCCc2ccccc2)ccc1N1CCCCC1', 'Clc1ccccc1N1N(Cc2ccccc2F)c2ccccc21', 'Cc1c([N+](=O)[O-])cccc1NC(=O)c1ccc(Cl)cc1', 'Clc1cccc2ccc(Br)cc12', 'Clc1ccccc1NCCn1nnc2ccccc21', 'Oc1ccc(F)cc1-n1cnc2ccc(Nc3ccccc3)cc2c1=O', 'Clc1ccc(COc2nc(Cl)ccc2Oc2ccccc2F)cc1', 'O=C(CN(Cc1ccccc1)c1ccccc1)NNc1cccc2ncccc12', 'Oc1cc(COC(=O)C2CCN(c3ccc(Cl)cc3)CC2)ccc1Oc1ccccc1', 'Clc1ccc(Br)cc1-c1ccccc1Cl', 'O=C(CNc1nncc2c1nc(N)n2C)N1CCN(C2CCCCC2)CC1', 'Clc1ccccc1-c1nn(CCCc2ccccc2)c2c1CCC2', 'Clc1ccccc1Cn1cccc(Br)c1=O', 'Clc1ccccc1Nc1ncccn1', 'Clc1ccccc1NCCc1ccc2c(c1)CCCN2Cc1ccccc1', 'Clc1ccccc1-c1csc(Cc2ccc(F)cc2)n1', 'O=C(C1CCCCC1)N(Cc1ccccn1)c1ccccc1', 'Clc1cccc(Br)c1NCc1ccccc1Oc1ccccc1', 'O=C(c1cnn(Cc2ccccc2)n1)N1CCN(Cc2ccccc2)CC1', 'Clc1ccccc1Oc1cc(Cl)ccc1Cl', 'O=C(CNc1nn(Cc2ccccc2)c2ccccc12)N1CCN(c2ncccc2F)CC1', 'Clc1cc(Br)ccc1-c1ccccc1F', 'Clc1cc(Nc2nc3ccccc3o2)ccc1OCc1ccccc1Cl', 'Clc1ccc(-c2cc(SCC(=O)NCC3CC3)ccc2Cl)cc1', 'Clc1ccccc1OCc1cccc(C(F)(F)F)c1', 'Clc1ccc(-c2nnnn2-c2ccc(F)cc2)cc1', 'Clc1ccccc1OCc1cc(OCc2ccc(F)cc2)nc2ccccc12', 'O=C(CS(=O)(=O)c1ccc2ccccc2c1)Nc1ccccc1', 'ClCc1ccccc1-n1cccc1', 'Clc1cccc(-c2cccc(I)c2)c1', 'Clc1ccc([C@@H](Cl)C[C@H](Cc2ccccc2)N2CCOCC2)cc1', 'Clc1ccccc1OCCNS(=O)(=O)c1ccccc1Br', 'Clc1ccc(Br)cc1OCc1cccc(F)c1', 'Clc1ccc(Br)cc1-c1ccccc1F', 'Clc1ccc(Br)cc1-c1cccnc1', 'COc1ncnc(N2CCN(c3ccccc3OCc3ccccc3)CC2)c1[N+](=O)[O-]', 'Clc1ccccc1CCC(=O)Nc1ccc2c(c1)CCC2', 'Clc1ccccc1OCCn1cccn1', 'Clc1ccc(COc2nccnc2Cc2ccccc2)cc1', 'O=C(Cc1cc2c(cc1Br)OCO2)N1CCN(Cc2ccccc2)CC1', 'Oc1ccc(/C=N/N2CCN(c3ccccc3)CC2)cc1', 'Clc1ccc(-c2nnnn2CCNc2ccc(Cl)cc2)cc1', 'O=C(c1cccc([N+](=O)[O-])c1)N1CCN(Cc2ccccc2)CC1', 'Clc1cccc(-c2cccc(OCc3ccccn3)c2)c1', 'Clc1ccccc1-c1n[nH]c2ccccc12', 'Clc1ccccc1-c1n[nH]c2ccccc12', 'Cc1noc(C)c1CCC(=O)N(C)Cc1ccccc1', 'O=C(/C=C1C(=O)N(c2ccc(Cl)cc2)c2ccccc21)c1ccccc1', 'Clc1ccccc1COC1CCC(CNC(=O)CCOc2ccccc2)CC1', 'O=C(COCCNc1ccc(Br)cc1)N1CCN(c2cccc[nH+]2)CC1', 'Clc1ccccc1Cc1ccc(Cl)c(F)c1', 'Clc1ccccc1Oc1cc(Cl)ccc1F', 'Clc1ccccc1-c1csc(COc2ccccc2)n1', 'Clc1cccc2cc(Br)ccc12', 'Clc1cccc2cc(Br)ccc12', 'Clc1cccc2cc(Br)ccc12', 'Clc1ccccc1OC1CN(C(=O)CNc2cccc(F)c2)C1', 'O=C([C@H](C(Cl)F)N1CCN(CC(=O)N2CCOCC2)CC1)Nc1ccccc1', 'O=C(CN(CCC(N)=O)C1CCCCC1)Nc1ccccc1', 'Clc1ccc(COc2ncccc2F)cc1', 'Clc1ccc(-c2nn(-c3ccccc3)cc2F)cc1', 'Clc1ccccc1OCCOc1ncccc1F', 'Clc1ccccc1NS(=O)(=O)c1ccccc1Br', 'Clc1cc(/C=C(/C#CCN2CCOCC2)c2ccccc2)ccc1Br', 'ClCc1ccccc1OCc1cccc2ncccc12', 'Clc1ccc(Br)cc1-c1ccncc1', 'Clc1ccc(Br)cc1-c1ccncc1', 'Clc1ccc(Br)cc1-c1ccncc1', 'Clc1ccccc1OCc1ccn(Cc2ccccc2Cl)n1', 'Nc1ncccc1Oc1ccccc1OCc1ccccc1Cl', 'Clc1ccccc1CNCCc1ccn(-c2ccccc2)n1', 'Clc1ccc(C(=O)Oc2ccc(F)c(F)c2)cc1', 'CNC(=O)c1ccc(OCCNC(=O)c2cnn(CC)c2)cc1', 'Oc1ccc(Oc2ccc(NCc3nc4ccccc4[nH]3)cc2)c(Cl)c1', 'Clc1ccccc1Nc1ncccc1Cl', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n\n', 'Clc1cccc2cc[nH]c12\n\n\n', 'Clc1cccc2cc[nH]c12\n\n', 'Clc1cccc2cc[nH]c12\n', 'Clc1cccc2cc[nH]c12', 'Clc1cccc2cc[nH]c12', 'Clc1cccc2cc[nH]c12', 'Clc1cccc2cc[nH]c12', 'Clc1cccc2cc[nH]c12', 'Clc1ccc(-c2nonc2NCc2nc(-c3ccncc3)no2)cc1', 'Clc1ccccc1Oc1cnc2ccccc2n1', 'OC[C@@]1(COc2cccc(F)c2)CO[C@@H]2[C@@H]1[C@@H]1[C@H](C1)CN2CC1CC1', 'Oc1ccc(F)cc1NS(=O)(=O)c1ccccc1', 'Oc1ccccc1COc1ccccc1Oc1ccc(F)cc1F', 'Clc1ccccc1Oc1cc(OC(c2ccccc2)c2ccccc2)cs1', 'Clc1ccccc1Oc1cc(F)ccc1F', 'O=C(CS(=O)(=O)c1ccc(Cl)cc1)NCCc1ccccc1F', 'Clc1ccccc1NS(=O)(=O)c1ccc([N+](=O)[O-])cc1', 'Oc1ccccc1OS(=O)(=O)c1ccccc1', 'Oc1ccccc1OS(=O)(=O)c1ccccc1', 'ClCc1cccc(-c2nc3ccccc3n2Cc2ccccc2)c1', 'O=C(Cc1nnc(-c2ccc(Br)cc2)o1)Nc1ccccc1F', 'Clc1ccc(-c2ccccc2CCO)cn1', 'Clc1ccccc1-c1cscc1-c1ccccc1', 'Clc1cccc(F)c1Oc1ccc(Br)cc1', 'Clc1ccccc1-c1cscc1-c1ccc(Br)cc1', 'O=C(CNCc1noc(COc2ccccc2)n1)N1CCOCC1', 'CNC(=O)c1ccc(CNC(=O)CCc2c[nH]cc2C(C)(C)C)cc1', 'O=C(c1ccc(Cl)cc1)N1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1', 'O=C(c1cccn2cc(Cl)cc12)N1CCC(C(=O)OCc2ccccc2)CC1', 'Clc1ccccc1Nc1ccnc(Oc2ccccc2)n1', 'O=C(CNS(=O)(=O)c1ccc(F)cc1)NC1CCCC1', 'Clc1ccccc1Oc1cnc2sc(-c3ccc(Br)cc3)nc2c1', 'Clc1ccccc1Oc1cc(-c2cccc(F)c2)ccc1O', 'Clc1ccccc1CNc1n[nH]c(-c2ccccc2)n1', 'O=C(CNC(=O)c1ccccc1)N/N=C/c1ccc2ccccc2c1', 'O=C(Cc1ccccc1)NCc1nncn1C1CC1', 'O=C(NCCNc1cccc2ccccc12)c1c[nH]c(=O)[nH]c1=O', 'O=[N+]([O-])c1c(F)cccc1NCC(=O)N1CCc2ccccc2C1', 'Clc1ccccc1NC(=O)c1ccc2c(c1)N(Cc1ccccc1C(F)(F)F)CCO2', 'O=C(c1c[nH]c2ncn(C(=O)c3ccccc3)c12)N1C[C@@H](COc2ccccc2F)C1', 'O=[N+]([O-])c1c(Cl)cc(C(=O)c2ccc(OCc3ccc(Br)cc3)cc2)cc1F', 'Clc1cc(F)cc(NCc2cccc(Cl)c2)c1', 'Clc1ccccc1Nc1cc(Cl)ccc1OCc1ccccc1F', 'Clc1cccc(F)c1-c1cccc(NCc2cccc(F)c2)c1', 'O=C(CS(=O)(=O)C[C@@H]1CN(Cc2ccccc2)CCO1)Nc1cccc2ccccc12', 'Clc1ccccc1COCCCn1cccn1', 'O=C(COCc1noc2ccccc12)Nc1ccc(Br)cc1', 'Oc1cc(Cn2nccc2-c2cccnc2)ccc1-c1cccnc1', 'Clc1ccc(CN[C@@H](c2ccccc2)C2CC2)cc1', 'Clc1ccc(CNS(=O)(=O)c2ccc(Br)cc2F)cc1', 'Clc1ccccc1COC(=O)c1ccc2c(c1)N(Cc1cc(F)cc(F)c1F)CC2', 'Oc1cccc2cc(-c3ccccc3)cnc12', 'O=C(c1cc(Cl)ccc1Cl)N(Cc1ccccc1)Cc1ccccn1', 'Clc1ccc(COCOc2ccc3ccccc3c2)cc1Cl', 'Clc1ccc(Br)c(F)c1-c1ccccc1', 'O=C(CN(Cc1ccccc1)c1ccccc1)Nc1cc(Cl)ccc1Cl', 'O=C(c1c[nH]nc1Nc1ccc(C(=O)NC2CC2)cc1)c1ccccc1', 'Clc1ccc(-c2cc(Cl)c(Cl)cn2)cc1', 'O=[N+]([O-])c1c(F)ccc(Nc2ccccc2)c1', 'CC1=C(C(=O)Nc2ccccc2)c2ccccc2C(=O)N1C', 'Clc1ccc(-c2cc(Br)ccn2)cc1', 'Clc1ccccc1-c1nccs1', 'Clc1ccccc1-c1nccs1', 'Clc1ccccc1-c1nccs1', 'Clc1cccc([S@@]C(=S)Nc2ccc3ccc(=O)n(Cc4ccccc4)c3c2)c1', 'Clc1ccc(CCO)c(NCc2ccccc2Br)c1', 'Clc1ccccc1N1N(CCc2ccccc2F)c2ccccc2C1', 'Clc1ccc(Br)c(Cl)c1\n', 'Clc1ccc(Br)c(Cl)c1', 'Clc1ccc(Br)c(Cl)c1', 'Clc1ccc(Br)c(Cl)c1', 'OCSc1ccccc1C(=O)NCc1ccccc1Oc1ccccc1', 'Clc1ccc(Br)cc1OCCn1ccccc1=O', 'Clc1ccc(-c2ccccc2)cc1C(=O)OCC(=O)Nc1cc(F)ccc1F', 'Clc1cccc([S@@]c2ccccn2)c1', 'NS(=O)(=O)Cc1ccc(Cl)cc1', 'Clc1ccccc1-c1nc2ccc(Cl)cc2s1', 'Clc1ccc([C@@H](Br)c2ccc(Cl)cc2)cc1', 'Clc1ccc(CNc2ccc3c(c2)OCO3)cc1', 'Clc1cccc(F)c1NCc1cccc(F)c1', 'Clc1ccccc1OCc1csc(-c2ccc(Cl)cc2)n1', 'Oc1cc(F)ccc1Oc1ccc(F)cc1', 'Clc1ccccc1Oc1cc(Cl)cc(Cl)c1', 'Clc1ccccc1Oc1cncc(-c2ccccc2)n1', 'Clc1cccc(Nc2cc(Br)ccc2F)c1', 'O=C(c1ccsc1)N1CCN(Cc2ccccc2)CC1', 'Clc1ccccc1-c1cncc(Br)c1', 'Oc1cccc(-c2csc(-c3ccccc3)n2)c1', 'Oc1cccc(CCCC(=O)c2ccccc2F)c1', 'Clc1cccc(SC2CCC2)c1', 'ClCc1csc(-c2ccccc2)n1', 'ClCc1csc(-c2ccccc2)n1', 'ClCc1csc(-c2ccccc2)n1', 'ClCc1csc(-c2ccccc2)n1', 'O=C(Cc1nc(-c2ccccc2)no1)NCc1cc(Cl)cc2ccccc12', 'O=C(CCNc1ccccc1N1CCOCC1)NCc1cccs1', 'Clc1ccccc1Oc1cc(F)c(Br)cc1', 'Clc1cccc(/N=C\\c2ccccc2)c1', 'Clc1cccc(/N=C/c2ccccc2)c1', 'Clc1ccc(Br)cc1S(=O)(=O)Nc1ccc(Cl)cc1', 'O=C(c1c[nH]cn1)Oc1cc(CNc2ccccc2)n(-c2ccccc2)n1', 'Clc1ccccc1Oc1cnc(F)cc1', 'O=C(CC[C@@H]1COc2ccccc2C1)NCCc1ccc(Cl)cc1', 'O=C(Cc1coc2nccc(NC(=O)Cc3ccccc3)c12)c1ccc(Cl)cc1', 'O=C(C1CCN(C(=O)c2ccccn2)CC1)N1CCCC1', 'Clc1cccc(Nc2cnc(-c3cnn(-c4nc(C(F)(F)F)ccc4Cl)c3)[nH]c2=O)c1', 'O=C(CCS(=O)(=O)c1cccc2ccccc12)OCc1ccccc1', 'Clc1ccc(-c2n[nH]c(Cc3ccccc3)n2)cc1', 'Clc1ccccc1Oc1cc(Br)ccc1F', 'Clc1ccccc1Oc1cc(Br)ccc1F', 'Clc1ccccc1Nc1nccn1Cc1ccccc1', 'Clc1ccccc1Cc1cn2ccccc2n1', 'O=[N+]([O-])c1ccnc2ccccc12', 'O=[N+]([O-])c1ccnc2ccccc12', 'O=[N+]([O-])c1ccnc2ccccc12', 'O=[N+]([O-])c1ccnc2ccccc12', 'O=[N+]([O-])c1ccnc2ccccc12', 'Clc1cccc(Nc2cc(Cl)ccc2O)c1', 'Clc1cc(Br)cnc1-c1ccc(Cl)cc1', 'Clc1ccccc1Nc1cnc(Oc2ccc(Cl)cc2)nc1', 'Oc1cccc2cc(/C=C/c3ccccc3)ccc12', 'Oc1ccccc1CCC1CCN(Cc2ccccc2-n2cc[nH+]c2C)CC1', 'O=C(Cc1nc2ccccc2[nH]1)NNc1ccccc1', 'O=C(c1c[nH]nc1CN(Cc1ccccc1)C1CC1)Nc1ccc2c(c1)OCO2', 'O=C(CCS(=O)(=O)Cc1ccc(Cl)cc1)NC1CCCC1', 'Clc1ccc(Br)cc1-c1ccnc(N2CCN(Cc3ccccc3Cl)CC2)c1', 'Clc1ccccc1CS(=O)(=O)c1ccccc1Cl', 'O=C(COc1ccc2ccccc2c1)NCc1ccc(Cl)s1', 'O=C(COc1cc(Br)ccc1Br)Nc1ccc(Cl)cc1', 'Oc1ccccc1N1C(=O)c2ccccc2C1=O', 'Clc1ccc(Cc2noc(Cc3ccncc3)n2)cc1', 'Oc1cccc(F)c1NC(=O)c1c[nH]c2ccccc12', 'Clc1ccccc1OCCCCn1cc(Br)cn1', 'Clc1ccccc1CNCc1ccc2c(c1)OCO2', 'Oc1ccccc1-n1cccn1', 'Clc1ccc(CCC(F)(F)F)cc1', 'Clc1ccccc1NS(=O)(=O)c1ccc(C(=O)N2CCOCC2)cc1', 'Clc1ccc(Br)cc1OC/C=C/c1ccccc1', 'C=CCOc1ccc(/C=N/NC(=O)Cc2ccc(C)cc2)cc1', 'O=C(/C=C/OCCOc1ccc(Oc2ccccc2)cc1)Nc1ccc(Cl)cc1', 'O=[N+]([O-])c1cccc2ncccc12', 'O=[N+]([O-])c1cccc2ncccc12', 'O=C(CS(=O)(=O)CCc1ccccc1)NCc1ccccc1F', 'Clc1ccccc1-c1cscc1-n1cc(-c2cccc(Cl)c2)cn1', 'Clc1ccc(-c2nonc2CN2CCN(c3ccccc3)CC2)cc1', 'Clc1ccccc1-c1noc(C2CCCCC2)n1', 'Oc1cc(OC(F)F)ccc1-c1ccccc1', 'O=C(c1c[nH]nc1N(c1ccccc1)c1cccc(F)c1)c1cccc(F)c1', 'ClCc1cc(Br)ccc1Oc1ccccc1', 'O=C(CSC(c1ccccc1)c1ccccc1)N1CCc2ccccc21', 'Oc1ccc(F)cc1NS(=O)(=O)c1ccc(Cl)cc1', 'Oc1ccc(F)cc1[C@H]1[C@H]([NH2+][C@H]2CCOc3ccc(Cl)cc32)OC1(CCc2ccccc2)c1ccc(F)cc1', 'Clc1cccc(-c2nc3cc(Cl)ccc3o2)c1', 'Clc1ccc(-c2cccs2)nc1', 'Clc1ccc(-c2cccs2)nc1', 'O=C(COCCO)Oc1ccccc1F', 'Oc1cc(-n2nnc(-c3ccc(Cl)cc3)n2)ccc1Oc1cccc(Cl)c1Cl', 'Oc1cccc(Nc2cc(Cl)ccc2Cl)c1', 'Oc1ccc(Oc2ccc(Cl)cc2)c(O)c1', 'Clc1ccccc1CN(C1CC1)S(=O)(=O)c1ccccc1Cl', 'Clc1ccccc1Oc1cc(F)ccc1O', 'Clc1ccccc1Oc1cc(F)ccc1O', 'NS(=O)(=O)c1ccc(CNC(=O)c2cccnc2)cc1', 'Clc1ccc(-c2ccccc2N2CCN(c3ccc(Cl)c(Cl)c3)CC2)cc1', 'Clc1ccccc1OCC(F)(F)F', 'Clc1ccccc1N1CCN(Cc2cccc(S(=O)(=O)N3CCCCC3)c2)CC1', 'COc1ncccc1F', 'O=C(/C=C/c1c(-c2ccccc2Cl)noc1-c1cccnc1)Nc1cccc(C(=O)[O-])c1', 'Clc1cccc2cc(O)ccc12', 'Clc1ccccc1NC(=O)CSc1nnnn1-c1ccccc1', 'O=C(CCCc1ccc2[nH]c(=O)[nH]c(=O)c2c1)Oc1ccccc1', 'Clc1cc(Br)ccc1Sc1ccccc1', 'Clc1ccccc1-c1csc(CN2CCN(Cc3ccccc3)CC2)n1', 'Clc1ccc(-c2cc(Cl)ccc2F)cn1', 'NS(=O)(=O)c1ccc(CNS(=O)(=O)c2cc(Br)ccc2Cl)cc1', 'O=C(COc1cc(Cl)c2c(c1)OCCO2)Oc1ccc(-c2ccccc2)cc1', 'Clc1ccc(-c2nnnn2-c2cc(OCc3cccnc3)ccc2[N+](=O)[O-])cc1', 'O=[N+]([O-])c1cc(Cl)ccc1Cl', 'Oc1ccc(F)c(Cl)c1', 'Clc1ccccc1Oc1cc(COc2ccc(Br)cc2)ccn1', 'Clc1ccccc1C(=O)Nc1ccc(Cn2nncc2C2CC2)cc1', 'Clc1ccccc1Oc1cnc2cc(C(=O)NCc3ccccc3)[nH]c2c1', 'Clc1ccccc1Oc1cc(-c2nnc(-c3cccs3)o2)ccc1F', 'Clc1ccccc1Oc1cnc(-c2ccccc2Cl)n1-c1ccccc1', 'Clc1ccc(Br)cc1COC(=O)c1cccc(F)c1', 'O=C(CNS(=O)(=O)c1ccc(F)cc1Cl)NCc1ccccc1', 'Clc1ccccc1C(=O)OCc1nc(Nc2ccccc2)no1', 'Oc1cc(Br)ccc1-c1cccc2ccccc12', 'O=[N+]([O-])c1cccc(NC2CC2)c1', 'Clc1ccc(C2CC2)cc1F', 'Clc1ccc(C2CC2)cc1F', 'Clc1ccc([C@@H](Br)Cc2ccccc2)cc1', 'Nc1n[nH]c2ccccc12', 'Clc1ccccc1COCCBr', 'Clc1ccccc1-c1cscc1-c1ccccc1C(=O)N[C@@H]1CCCc2ccccc21', 'N#Cc1cccc(NC(=O)COc2ccc(Br)cc2)c1', 'Clc1ccc(COc2nc(C3CC3)n(-c3ccccc3)n2)cc1', 'Clc1ccccc1Cc1nc(-c2ccccc2Cl)no1', 'Clc1ccccc1Oc1cc(Cl)c(Cl)cc1Cl', 'Clc1ccccc1Oc1cc(Cl)c(Cl)cc1Cl', 'Clc1ccccc1Nc1cc(Cl)ccn1', 'O=C(CN(c1cnn(C(C)C)n1)c1ccccc1)N1CCN(C2CC2)CC1', 'Clc1ccc(CO)c(Cl)c1', 'Cc1c(Br)cccc1NC(=O)NCC(C)C', 'Clc1ccsc1-c1ccc(Cl)cc1', 'Clc1ccc(COCOC2CCCC2)cc1', 'Clc1ccc(CN[C@@H](C(=O)c2ccccc2)c2ccccc2)cc1', 'Clc1ccccc1Oc1cc(OCc2ccccc2)ncn1', 'Clc1ccc(C(F)(F)F)nc1OCCCOc1ccccc1', 'Clc1ccc(Br)cc1Sc1ccccc1', 'Clc1ccc(Br)cc1Sc1ccccc1', 'Clc1cccc(/C=N/NC(=S)NCc2ccccc2)c1', 'Clc1ccccc1OCCCc1nnc2n1CCC2', 'O=C(Cc1nnnn1Cc1ccccc1)Nc1ccncc1', 'Clc1ccccc1OCc1ccc(Br)c(N2CCOCC2)c1', 'O=C(COc1cc(Br)c2ccccc2n1)Nc1ccccc1', 'Clc1ccccc1CNC(=O)N1CCCc2ccccc21', 'O=C(CS(=O)(=O)c1ccc(Cl)c(Cl)c1)Nc1ccc(F)cc1', 'Oc1ccc(Oc2ccc(Br)cc2)cc1O', 'Clc1ccccc1-c1noc(Cc2ccccc2F)n1', 'Clc1ccccc1-c1nc(-c2ccncc2)no1', 'Oc1cccc(Oc2nc(-c3cccs3)nc3ccccc23)c1', 'Clc1cccc(Cn2c3ccccc3nc2SCc2ccc(Br)cc2)c1', 'Clc1nnn(-c2ccccc2)c1-c1ccccc1', 'NS(=O)(=O)c1cc(NC(=O)COc2ccc(Br)cc2)ccc1O', 'Clc1ccc(-c2csc3c(Cl)cccc23)cc1', 'Oc1cccc(N2CCOCC2)c1', 'Clc1ccccc1-c1csc(-c2cccc(Br)c2)n1', 'O=C(CCc1ccccc1)N1CCC(NC(=O)Nc2ccnc(F)c2)CC1', 'Clc1ccccc1NCCS(=O)(=O)c1ccccc1', 'OCCC(=O)N1CCN(C(=O)c2cccs2)CC1', 'Clc1ccc(Br)cc1NCc1ccncc1', 'O=C(COc1cc(F)ccc1N1CCCC1=O)c1ccccc1', 'Clc1ccc(-c2nnnn2CC(=O)N2Cc3ccccc32)cc1', 'O=C(CCc1ccccc1)Nc1ccc(-n2ncn(C)c2=O)cc1', 'Clc1ccc(C(=O)c2ccc(O)cc2F)cc1', 'Clc1ccc(COc2nc(CN3CCC(c4ccccc4)CC3)no2)cc1', 'Clc1ccccc1Oc1cc(Br)cc2ccccc12', 'Clc1ccccc1Oc1cc(O)ccc1F', 'CNc1nn(Cc2ccc(Cl)cc2)c(N2CCCC2)c1/C=C/c1ccc(O)cc1', 'Clc1ccc(Cc2noc(-c3ccc(O)cc3)n2)cc1', 'Clc1ccc(Br)c(F)c1CN(Cc1ccccc1)Cc1ccco1', 'O=C(CCS)Nc1ccccc1C(=O)Nc1ccccc1C(F)(F)F', 'Clc1ccccc1Oc1cnc2nn(-c3ccccc3)nc2c1', 'Clc1ccccc1Oc1cc(Cl)c(CO)cc1', 'Clc1ccc(Br)cc1NCC(=O)N1CCC[C@@H](c2[nH+]ccn2CC2CCCC2)C1', 'O=[N+]([O-])c1cc(Br)ccc1Oc1ccc(Cl)cc1', 'Clc1ccccc1NCCCO/N=C/c1ccccc1[N+](=O)[O-]', 'Oc1cc(Cn2cccn2)ccc1OCc1ccc(F)cc1', 'Clc1ccccc1NS(=O)(=O)Nc1ccc(CC2CC2)cc1', 'Oc1cccc(-c2cc(C3CC3)nc3ccccc23)c1', 'CNC(=O)CCNC(=O)c1ccc(F)cc1F', 'Oc1cccc2cc(O)ccc12', 'Oc1cccc2cc(O)ccc12', 'O=C(CNCc1csc2ccccc12)N1CC2(CCCCC2)Oc2ccccc21', 'Clc1ccccc1Cc1cc(F)ccc1O', 'Clc1cccc(F)c1COc1cccc(Cl)c1OCCC1CC1', 'Clc1ccccc1C(=O)N(Cc1ccncc1)Cc1ccccc1C(F)(F)F', 'Clc1ccccc1NS(=O)(=O)c1cccc(C(F)(F)F)c1', 'Clc1ccc(-c2nn(-c3ccc(F)cc3)cn2)cc1', 'Clc1ccc(-c2nn(-c3ccc(F)cc3)cn2)cc1', 'CC(C)c1cc(F)ccc1S(=O)(=O)Nc1ccc(Cl)c(Cl)c1', 'Clc1ccc(Br)cc1CCNc1ccccn1', 'O=[N+]([O-])c1ccc(Sc2ccc(Cl)cc2)cc1', 'O=C(CNS(=O)(=O)c1ccc([N+](=O)[O-])cc1)NCc1ccccc1', 'Clc1ccccc1C(=O)N/N=C\\c1ccccc1O', 'O=C(CNc1nn2c(=O)cc(-c3ccc(Cl)cc3Cl)cc2s1)Nc1cccc2ncccc12', 'Clc1cc(/C=C/Nc2ccc(OCc3ccccc3)cc2)ccc1Cl', 'Oc1ccc(/C=N/Nc2ncnc3ccccc23)cc1', 'Clc1ccccc1Oc1cnccn1', 'Clc1ccccc1Oc1cnccn1', 'Clc1ccccc1Oc1cnccn1', 'Clc1ccccc1NC(=O)/C=C/c1ccc([N+](=O)[O-])cc1', 'Clc1ccc(Br)cc1NCC(=O)N1CCCC1', 'Clc1ccc(Br)cc1-c1cnc2ccccc2n1', 'Clc1ccccc1Cc1c(O)ccc2cccnc12', 'ClC[C@@H]1CN(Cc2cc(F)ccc2F)CC1', 'Cc1ccccc1OCC(=O)N1CCO[C@@H](c2cccc(C(F)(F)F)c2)C1', 'CCCCN(Cc1ccco1)C(=O)Nc1ccc(Cl)cn1', 'O=C(CNCc1nc2ccccc2c(=O)[nH]1)c1ccc(Br)cc1', 'Clc1ccc(CCOC2CCOCC2)cc1', 'Clc1ccc(Br)cc1CCNC(=O)c1cccs1', 'O=C(CCc1nc(-c2ccccc2)cs1)Nc1ccc(F)cc1Cl', 'ClCc1ccnn1Cc1ccccc1Oc1ccccc1F', 'Clc1ccccc1-c1nn(Cc2ccccc2)nc1N1CCC(c2ccccc2)CC1', 'Clc1ccccc1-c1nc(C(=O)CCc2cccc(-c3ccccc3)c2)n[nH]1', 'Clc1cc(/C=C/C(=O)N2CCN(Cc3ccccc3)CC2)ccc1O', 'Clc1ccccc1Oc1cnc(Nc2ccccc2F)cn1', 'O=C(Cc1c([O-])c2ccccc2oc1=O)Nc1ccc(Oc2ccccc2)cc1', 'Clc1ccc(-c2cc(Cl)ncn2)cc1', 'Clc1ccccc1Nc1cncc(-c2ccc(F)cc2)n1', 'Clc1cccc(Nc2cc(F)ccc2Oc2cccc(F)c2)c1', 'Oc1ccc(F)cc1C(=O)Nc1ccc([N+](=O)[O-])cc1', 'Clc1cc(F)c(Cl)cc1Oc1ccccc1F', 'Clc1ccc(Br)cc1S(=O)(=O)N1CCCC1', 'O=[N+]([O-])c1c(F)cccc1NCCOCc1ccccc1', 'O=C(CNCc1cccs1)Nc1ccc(Cl)c(Cl)c1', 'Clc1ccccc1Oc1cncc(Oc2ccc(Cl)cc2)n1', 'O=C(CSC(=S)N1CCc2ccccc21)NCCc1ccccc1', 'Clc1ccccc1Nc1cc(NS(=O)(=O)c2ccc(F)cc2)c(Cl)cc1Cl', 'C=CCNC(=O)c1ccc(C)c(OC)c1', 'O=C(COc1ccccc1C(=O)[O-])NCc1ccccc1', 'NS(=O)(=O)CCNS(=O)(=O)c1ccccc1', 'COc1ccccc1C(=O)OCCC(=O)N1C[C@H](C)C[C@@H](C)C1', 'NS(=O)(=O)Nc1cccc(NC(=O)c2cccc(F)c2)c1', 'Clc1ccccc1Oc1cc(Br)ccc1O', 'Clc1ccccc1Oc1cc(Br)ccc1O', 'Clc1ccc(-c2nn([C@@H](Cc3ccccc3)c3ccc(F)cc3F)nn2)cc1', 'Oc1ccc([N+](=O)[O-])c(NCc2ccccc2)c1', 'Clc1ccccc1O/Cn1nc(-c2ccccc2F)nn1', 'O=C(COc1ccc(Cl)cc1)Nc1ccc(F)c(C(F)(F)F)c1', 'Clc1ccc(-c2nnnn2-c2ccccc2)cc1Cl', 'Clc1ccccc1-c1nn(-c2ccccc2)c(C(=O)N(Cc2ccccc2)C2CC2)c1C(F)(F)F', 'Clc1cccc([S@@]c2cccc(Br)c2)c1', 'Clc1ccccc1CNC[C@H](O)c1ccccc1', 'Clc1ccccc1Nc1ccn(-c2ccc(Br)cc2)n1', 'Clc1ccc(CCNCc2cc(F)cc3c(O)cccc23)cc1', 'O=C(c1ccc(NC(=O)c2sc(N3CCOCC3)nc2C2CC2)cc1)N1CCC[C@@H]1c1ccc(Cl)cc1', 'O=[N+]([O-])c1cc(Cl)ccc1CCOc1ccccc1F', 'Clc1ccc(CNS(=O)(=O)c2c(Cl)cccc2Cl)cc1', 'Clc1ccccc1Nc1cc(F)cc(Br)c1', 'Oc1ccc(Oc2cc(Br)ccc2F)cc1', 'Clc1ccccc1CNC[C@@H](O)COc1ccccc1', 'Clc1ccc(Br)cc1-c1cc(Cl)cc2ccccc12', 'O=C(/C(COc1ccccc1)c1ccccc1)[C@@H]1COc2ccccc2O1', 'Clc1ccccc1N1N(CCNC(=O)c2cccc3ccccc23)CC1', 'Clc1ccccc1Oc1cc(Cl)ccn1', 'Clc1ccc(COc2ncc(Cl)cc2CO)cc1', 'Oc1ccc(F)cc1OCCNC(=O)c1ccccc1Cn1cccn1', 'Clc1cc(Br)cc(-c2ccncc2)c1', 'Clc1nnc(-c2ccccc2)n1COc1ccccc1', 'Clc1ccccc1N1N(C(=O)Cc2ccccc2F)CCC1', 'Clc1ccccc1C(=O)OCC(F)(F)F', 'Clc1ccccc1-c1nc(N2CCCC2)nc2ccccc12', 'O=C([C@H](Cl)Cc1ccccc1)c1ccccc1', 'Clc1nnn(-c2ncn(-c3ccccc3)n2)c1-c1ccccc1', 'Clc1ccccc1CNc1ccc(-n2cccn2)nc1', 'Clc1ccccc1NCc1ccnc(OC2CCC2)c1', 'O=C(c1c[nH]c2ncn(-c3ccccc3)c12)c1ccccc1', 'O=C(c1ccsc1)c1cn(-c2ccc(F)cc2)nc1-c1ccccc1[N+](=O)[O-]', 'Clc1ccccc1Oc1cc(S(=O)(=O)N2CCCC2)ccc1Cl', 'O=C(COCCCCl)N1CCC[C@H]1c1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(F)c1C(=O)NCCCc1ccccc1Cl', 'Clc1cccc(F)c1Cn1ccc(-c2ccc(Cl)cc2)n1', 'Clc1ccccc1NCC(=O)N1CCN(S(=O)(=O)c2ccc(F)cc2)CC1', 'Clc1cc(Br)cnc1Oc1ccccc1F', 'Clc1ccc(-c2ccccc2C2(c3ccc(Cl)cc3)CCOCC2)cc1', 'Clc1cc(Cl)ccc1-c1ccccc1CCNC(=O)N1CCOCC1', 'Clc1ccc(-c2ccccc2)nc1SCC1CCCC1', 'Oc1cc(-n2cc(-c3ccccc3)nc2-c2ccc(Br)cc2F)ccc1F', 'Clc1cccc(Br)c1Cl\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n\n\n\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n\n\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n\n', 'Clc1cccc(Br)c1Cl\n\n\n', 'Clc1cccc(Br)c1Cl\n\n', 'Clc1cccc(Br)c1Cl\n', 'Clc1cccc(Br)c1Cl', 'Clc1cccc(Br)c1Cl', 'CNC(=O)NC(=S)N(CCCO)Cc1ccccc1', 'O=C(c1ccccn1)Nc1cccc(NC(=O)c2ccco2)c1', 'Clc1cc(Cl)ncc1Cc1ccccc1', 'Clc1ccccc1Oc1cc(Br)cc(F)c1', 'Clc1ccccc1Oc1cnc(CCn2ccccc2=O)cc1', 'Oc1cccc(CCl)c1', 'Oc1ccc(/C=C/Nc2cccnc2)cc1', 'O=C([C@H](c1c(F)cccc1Cl)[NH+]1CCN(c2cccc(Cl)c2Cl)CC1)C1CCCC1', 'Clc1ccccc1Oc1cnc2snc(-c3ccc(Br)cc3)c2c1', 'Clc1ccccc1OCc1cc(F)cc2ccc(F)cc12', 'Oc1ccc([C@H]2CCOCC2)cc1', 'Clc1ccc(-c2cc(O[C@H](Cc3cccc(F)c3)c3ccccc3)n[nH]2)cc1', 'Clc1ccc(-c2cc(Br)ccc2N2CCCC2)cc1', 'Clc1ccc(-c2nnnn2C2CC2)cc1', 'Clc1ccc(-c2nnnn2C2CC2)cc1', 'O=C(CNS(=O)(=O)/c1ccc2ccccc2c1)N1CCN(c2ccc(F)cc2)CC1', 'ClCc1cc(Br)ccc1Oc1ccccc1F', 'Clc1ccccc1O/Cn1nnc2ccccc2c1=O', 'Clc1cccc(Nc2cc([N+](=O)[O-])ccc2Cl)c1', 'Oc1ccc(Oc2cc(F)cc(F)c2)cc1', 'O=C(Cc1c(N)nc(-c2ccccc2)o1)c1ccccc1', 'Clc1ccc(-c2ccccc2N2c3ccc(F)cc3C(=O)N2C2CCCCC2)cc1', 'O=[N+]([O-])c1c(Cl)ncnc1N[C@@H](c1ccccc1)c1ccccc1', 'Clc1ccccc1Oc1cncc(Br)c1', 'Oc1cccc(C(=O)Nc2ccccc2-c2ccco2)c1', 'O=C(CC(=O)Nc1ccc(Cl)cc1F)Nc1cccc(N2CCCC2)c1', 'ClCc1cccc([N+](=O)[O-])c1', 'Clc1ccccc1Nc1cnc(NCc2cccnc2)nc1', 'Clc1cc(/C=C/Cc2ccccc2F)ccc1Cl', 'Clc1ccccc1Oc1cc([N+](=O)[O-])ccc1Cl', 'Clc1ccc(Br)cc1-c1ccc(Cl)cc1F', 'Clc1ccccc1CS(=O)(=O)CC(=O)Nc1ccc(Br)cc1', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccccc1', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccccc1', 'Clc1ccc(CN2CCc3ccccc3C2)c(F)c1', 'Clc1ccc(COC[C@@H]2CCCN2C(=O)c2ccccc2Cl)cc1', 'Oc1cc(Br)cc2ccccc12', 'Clc1ccccc1Oc1cc(S(=O)(=O)N2CCCCC2)ccc1Cl', 'Clc1ccc(-c2nnnn2-c2cccc(Cl)c2)cc1', 'Clc1ccc(Br)cc1-c1cc(F)ccc1F', 'Clc1cccc2cc([C@H](Cl)c3ccc(OCc4ccccc4F)cc3)sc12', 'Clc1ccc(CN2C(=O)NC3(CCN(CC(=O)N4CCOCC4)CC3)C2=O)cc1', 'O=C(CNC(=O)Cc1ccccn1)NCC1(c2ccc(F)cc2)CCC1', 'Clc1ccc(Br)cc1OCC1CCCCC1', 'Clc1cccc(Nc2cc(F)cc(Br)c2)c1', 'O=[N+]([O-])c1cccc2[nH]ccc12', 'O=[N+]([O-])c1cccc2[nH]ccc12', 'O=[N+]([O-])c1cccc2[nH]ccc12', 'O=[N+]([O-])c1cccc2[nH]ccc12', 'Clc1ccccc1[C@H]([NH2+][C@@H]1CC[C@@]1(O)COc1ccccc1)c1ccc(Br)o1', 'O=C(CNc1cnn(CN2CCCC2)c1)NCc1nc2ccccc2c2c1CCCC2', 'Clc1cc(NS(=O)(=O)c2ccccc2)c2ccccc2n1', 'Clc1ccccc1COCCc1cccc(-c2ccco2)c1', 'Clc1ccccc1Oc1cc(F)c(Cl)cc1Cl', 'COC(=O)N1CC[C@@H]1CC(=O)Nc1ccccc1Oc1ccccc1F', 'Oc1cccc(NCc2cccc(Cl)c2Cl)c1', 'Cc1cc(Br)ccc1NC(=O)c1ccc(F)cc1Br', 'Clc1ccccc1Oc1cc(F)c(F)cc1F', 'Clc1ccccc1CN(Cc1ccccc1)C(F)(F)F', 'Clc1ccccc1C(=O)Nc1cnc2ccccc2c1O', 'Clc1ccccc1NS(=O)(=O)c1c[nH]c2ccccc12', 'O=[N+]([O-])c1ccn(Cc2ccccc2)c1', 'Clc1ccc(Cc2noc(C3CC3)n2)cc1', 'Clc1ccccc1C(=O)OC1(c2ccc(Cl)cc2)CC1', 'Clc1ccccc1Nc1ncnc(N2CCC(c3ccc(F)cc3)CC2)n1', 'Clc1ccc([C@@H](Br)CSCc2ccccc2)cc1', 'O=C(CNC(=O)COc1ccc(Cl)cc1F)NCc1cccs1', 'Clc1ccccc1Cc1noc(CCN2CCOCC2)n1', 'Clc1cccc([S@@]c2c[nH]c3ccccc32)c1', 'Oc1cc(COCc2ccncc2)ccc1O', 'OC[C@@H](NCc1ccc(OCCO)cc1)c1ccccc1', 'Clc1ccccc1Oc1cc(F)c(N2CCCC2)c(I)c1', 'Clc1ccc(Br)c(S[C@H](C2CCCCC2)C2CC2)c1', 'Clc1ccc(Cc2n[nH]c(-c3ccco3)n2)cc1', 'Oc1ccc(Oc2ccc3ccccc3n2)cc1Cl', 'Clc1ccccc1NC(=S)NCc1ccc2c(c1)OCCO2', 'Oc1cccc(-c2ccc(C(=O)Nc3cccc(F)c3)nc2)c1', 'O=C(Cc1nn(-c2ccccc2)c2c1CCC2)c1ccccc1', 'Clc1ccccc1Nc1ncc(Br)cn1', 'Clc1ccccc1OCc1ccc2c(c1)CCC(=O)N2', 'Oc1cccc(Br)c1Oc1ccccc1F', 'O=C(c1c[nH]c2cc(Cl)ccc12)c1ccc(Br)cc1', 'Clc1ccccc1OCC(=O)Nc1ccc(C(=O)[O-])cc1', 'O=[N+]([O-])c1ccn(CCc2ccccc2Cl)c1', 'OCS(=O)(=O)c1ccc(N2CCN(c3ccccc3)CC2)cc1', 'O=C(Cc1nc2ccccc2c(=O)[nH]1)Nc1ccc2ccccc2c1', 'Clc1ccc(-c2cc(Br)cs2)cc1', 'Clc1ccc(C(CCBr)OCc2ccccc2)cc1', 'Clc1ccc(-c2nn(-c3ccc(Br)cc3)c(=O)n2Cc2cccc(F)c2)cc1', 'Oc1cccc(-c2ccnc(NCCOc3ccccc3)c2)c1', 'ClCc1noc(-c2ccccc2)n1', 'Oc1ccccc1/N=C/c1ccccc1COc1ccccc1Cl', 'Clc1ccccc1NS(=O)(=O)c1cc(Cl)cc(Cl)c1', 'O=C(c1cccnc1)N1CCc2cc(Cl)ccc21', 'Clc1cccc(F)c1-c1cc(Cl)cc(Cl)c1', 'Clc1ccc(Br)cc1-c1cc2ccccc2cc1F', 'O=C(C1CCN(C(=O)C2CCCC2)CC1)N1CCc2ccccc2C1', 'Clc1cccc(CNc2ncnc3sc(-c4ccccc4)cc23)c1', 'Clc1ccc(C(F)(F)F)cc1F', 'Clc1ccccc1NS(=O)(=O)c1cccc(S(=O)(=O)N2CCCCC2)c1', 'Oc1ccc(/C=N/c2ccccc2Cl)cc1', 'O=C(CNc1nnc(Cl)c2ccccc12)Nc1ccccc1Cl', 'Clc1ccc(COc2nc([N+](=O)[O-])ccc2N2CCCCC2)cc1', 'Oc1ccc(F)cc1Oc1ccc(NCc2ccc(Cl)nc2)cc1', 'O=C(c1ccc(Br)cc1)N1CCC(OC(F)F)CC1', 'Clc1ccccc1-c1cnn(-c2cc(F)cc3ccccc23)c1O', 'Clc1ccccc1OC1CCN(c2ncccn2)CC1', 'Clc1cccc(S(=O)(=O)Oc2ccccc2Br)c1', 'O=C(CC1(C2=CC(=O)c3ccccc32)CC1)N1CCCC1', 'Oc1cccc([C@H]2OCCN2C(=O)CSCc2ccccc2)c1', 'Clc1cc(N2CCN(Cc3cccc(F)c3)CC2)ccn1', 'Clc1cc(Br)cc(CCOCc2ccc(F)cc2)c1', 'Clc1ccccc1NC(=S)NCc1cccc(C(F)(F)F)c1', 'O=C(COc1ccc(F)c(F)c1)N1CCC(Oc2ccc(C(C)=O)cc2)CC1', 'Clc1ccccc1Nc1c2onc2ncn1Cc1ccccc1', 'O=C(CCCn1cnnn1)N1CCN(Cc2ccc(F)c(F)c2)CC1', 'O=C(COCc1cc(F)ccc1F)Oc1ccccc1', 'CN(c1ccccc1)S(=O)(=O)Cc1ccccc1N', 'Oc1cccc(-c2cccc(NC(=O)c3ccc(F)c(S(=O)(=O)N4CCCC4)c3)c2)c1', 'O=[N+]([O-])c1c([O-])n(-c2ccccc2)c2ccccc12', 'COCc1cc(OC)ccc1NC(=O)N(C)Cc1ccc(F)cc1', 'O=[N+]([O-])c1c(F)cc(N(C(=O)CN2CCOCC2)c2nc3ccccc3s2)c2ccccc12', 'Clc1ccc(Br)cc1Sc1ccc(Br)cc1', 'Clc1ccccc1C1(Oc2ccccc2)CC1', 'Clc1ccccc1NS(=O)(=O)c1ccc(Br)c([N+](=O)[O-])c1', 'Clc1ccc(-c2cc(F)c(O)cc2)c(F)c1', 'Oc1ccc(/C=C(/Nc2cccc(F)c2)c2ccccc2)cc1', 'Oc1ccc(Oc2ccc(/C=N/NC[C@@H](c3ccccc3)c3ccco3)nc2Cl)cc1', 'O=C(CSCCN1CCOCC1)c1cc(Cl)c(Cl)cc1Cl', 'Oc1cc(F)ccc1SCC(=O)N1CCCC1', 'Nc1nonc1-c1ccccc1', 'Nc1nonc1-c1ccccc1', 'Clc1ccc(-c2n[nH]c(C(=O)N3CCCC3)c2)cc1', 'Clc1ccccc1-c1ncc[nH]1', 'Clc1ccccc1-c1ncc[nH]1', 'CCS(=O)(=O)c1ccc(NC(=O)c2cccc(C#N)c2)cc1', 'O=C(Cc1cc(Cl)ccc1F)Nc1ccccc1-c1ncccn1', 'O=[N+]([O-])c1c(F)cc(NCCCSC2CC2)cc1', 'Oc1cc(Oc2ccc(CS(=O)(=O)Cc3ccc(Cl)cc3)cc2)ccc1F', 'COc1ncnc(NCCc2ccccc2)c1Br', 'Clc1ccccc1OCc1cc([N+](=O)[O-])ccc1Br', 'Clc1cccc(Nc2cc(S(=O)(=O)N3CCCCC3)ccc2Cl)c1', 'Clc1cccc(F)c1-c1cccc(C(F)(F)F)c1', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n\n', 'Clc1cccc2cc[nH]c(=O)c12\n', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc2cc[nH]c(=O)c12', 'Clc1cccc([S@@]CCS(=O)(=O)c2ccc(F)cc2)c1', 'Clc1ccccc1Oc1cnc2nnc(-c3ccncc3)n2c1', 'Clc1ccccc1OCc1cccc(COCCO)n1', 'Clc1ccc(Cc2cccc(O)c2)nc1', 'O=C(Cc1c(Cl)cccc1Cl)N(c1ccccn1)Cc1ccccc1', 'Clc1ccccc1Cc1c(NC(=O)N2[C@H]3CC(Cc4ccccc4)CC3=CO[C@H](c3ccc(F)cc3)N2)ccc(Cl)c1O', 'Clc1ccccc1OCCCCNc1ncnc2c1CCC2', 'O=C(COc1ccc2cc([N+](=O)[O-])ccc2c1)Nc1cccc(Cl)c1', 'Clc1cc(CNC[C@H](c2ccccc2)c2ccccc2)c(F)cn1', 'Clc1ccccc1O/N=C/c1ccc(-c2ccccc2)o1', 'O=C(CN(c1ccccc1)S(=O)(=O)/C=C/c1ccccc1)NCc1ccccc1', 'Clc1ccc(Br)cc1-c1ccccc1N1CCOCC1', 'Clc1ccccc1NCc1ccc(Cl)o1', 'O=C(c1ccsc1)Nc1ccccc1OC1CCCC1', 'O=C(COc1ccc2c(c1)CCC(=O)N2)N1CCN(c2ccccc2)CC1', 'Clc1ccc(CCO/N=C(\\C)CCc2ccccc2)cc1', 'OCc1ccc(-c2csc(-c3ccccn3)n2)cc1', 'Clc1cccc([S@@]CC(F)F)c1', 'O=C(c1cccs1)c1ccc(N2CCN(c3cccc(Cl)c3)CC2)cc1', 'Clc1ccc(-c2nonc2CN2CCN(c3ccccc3Cl)CC2)cc1', 'Clc1ccc(-c2nnnn2CC2CC2)cc1', 'Oc1cc(F)ccc1-c1cccc(S(=O)(=O)N2CCN(c3ccccc3)CC2)c1', 'Oc1ccc(F)cc1NC(=O)Cc1ccc(F)c(Cl)c1', 'O=C(c1ccsc1COc1ccccc1)N1CCCc2ccccc21', 'Clc1ccc([C@@H](NNC(=O)c2ccc(N3CCCCC3=O)cc2)N2CCCCC2)cc1', 'Clc1cccc(NCCNC(=O)c2cccc(F)c2F)c1', 'O=[N+]([O-])c1ccc2ccc(Br)cc2c1', 'Clc1ccc(Br)cc1-n1cnc2c(F)cccc21', 'Clc1cc(Br)c(-c2ccccc2Cl)cc1Cl', 'Clc1ccccc1C(=O)N1CCCc2ccc(F)cc21', 'Clc1ccccc1Cn1cnnc1SCc1ccccc1', 'Oc1ccccc1C1(O)CCCCC1', 'O=C(CN(CC1CC1)C1CC1)Nc1cccc2cccnc12', 'Clc1ccc(-c2cc(F)c(Cl)cc2O)cc1', 'Clc1cc(F)c(OCc2ccccc2F)cc1C1CC1', 'Clc1ccc(Br)cc1OC(=O)c1cccnc1', 'Clc1cccc2ncnc(Sc3ccccc3)c12', 'Nc1nc2ccccc2n1CCOCc1ccccn1', 'Clc1ccccc1Cn1nc2cc(-c3ccccc3)ccn2c1=O', 'Oc1cc(-n2cn/cn2)ccc1-c1ccccc1', 'Clc1ccccc1Nc1cc(-c2ccccc2)oc1N1CCN(c2ncccn2)CC1', 'Clc1ccc(COCc2nnc(-c3ccccc3OC(F)(F)F)o2)cc1', 'Clc1ccccc1Oc1cnc(COc2ccccc2)[nH]1', 'Clc1ccccc1CCCSc1nc2ccccc2[nH]1', 'Clc1cc(CN2CCN(Cc3ccccc3)CC2)cc(OCC(F)(F)F)c1', 'Clc1ccccc1Nc1nnnn1-c1ccccc1', 'Clc1ccccc1OCCS(=O)(=O)C1CC1', 'CNC(=O)c1ccc(NC(=O)CNC(=O)c2ccccc2Br)cn1', 'O=C(Cc1cn(-c2ccc(F)cc2F)nn1)NCc1ccccc1', 'Clc1ccc(-c2cc(-c3csc4ccccc34)[nH]n2)cc1', 'Oc1ccc(Oc2ccc(NCc3ccco3)cc2)c(Cl)c1', 'O=C(/C=C/O)c1ccccc1', 'O=C(CNS(=O)(=O)/c1c(Cl)cccc1Cl)NCCc1ccccc1', 'ClCc1ccccc1OC(F)F', 'Clc1ccccc1Cc1ncn(-c2ccccc2)n1', 'O=C(CC[C@@H]1CN(C2CCCC2)CCN1Cc1ccccc1)Nc1ccccc1F', 'Clc1ccc(C(F)(F)F)cn1', 'Clc1ccc(Cc2noc(-c3ccc([N+](=O)[O-])cc3)n2)cc1', 'O=C(CN(Cc1ccccc1)c1ccccc1)Nc1nncs1', 'Clc1ccc(CO)c(C(=O)N2CCCCC2)c1', 'ClCc1cc([N+](=O)[O-])ccc1OCc1ccccc1F', 'O=C(C1CCN(C(=O)CN2CCOCC2)CC1)Nc1ccc([N+](=O)[O-])cc1', 'Clc1ccc(-c2ccccc2CC2(O)COC2)cc1', 'O=C(/C=C1/cc[nH]1)NCC1(N(C)C(=O)c2ccccc2)CCCCC1', 'O=[N+]([O-])c1cc(Cl)c(Cl)cc1Oc1ccc2ccccc2c1', 'O=[N+]([O-])c1ccc2occc2c1-c1ccccc1', 'Clc1ccc(-c2ccccc2)c(N/C=C2\\C(=O)N(c3ccc(Cl)cc3)C2=O)c1', 'Clc1ccc(-c2cc(F)c(Cl)cc2N)cc1', 'OCc1nc(Cc2ccccc2)no1', 'Clc1ccc([C@@H](NCCC2CCOCC2)c2ccccc2)cc1', 'O=C(Cc1nonc1CO)NCCc1ccccc1', 'Oc1cccc(Br)c1OC1(c2ccccc2)CCCC1', 'O=C(c1cc(Cl)ccn1)N1CCOCC1', 'O=C(C1=CCCN1c1cccc2cnccc12)Nc1ccc2c(c1)OCO2', 'Clc1ccc(-c2n[nH]c(Nc3ccc(Cl)cc3)n2)cc1', 'Oc1cccc(OCCc2cccnc2)c1O', 'Oc1ccc(Oc2nc(COc3ccccc3)no2)cc1', 'Clc1cc(Br)c(Nc2ncnc3ccccc23)cc1/C=N/c1ccccc1', 'Oc1ccccc1Oc1ccc(N2CCOCC2)cc1Cl', 'ClCc1cn(Cc2ccccc2)nc1-c1cccnc1', 'NS(=O)(=O)Cc1ccc([N+](=O)[O-])cc1', 'Clc1ccc(Br)cc1/C=N/NC(=O)C(c1ccccc1)c1ccccc1', 'Clc1ccc(-c2cccs2)nc1CNc1ccccc1', 'Clc1ccc(Cc2nnnn2-c2ccccc2F)cc1', 'Clc1ccc(Br)c(NC2CCCCC2)c1', 'O=C(CNc1nn2cccc2c2ccccc12)Nc1cccc(-c2nncn2C2CC2)c1', 'Clc1ccc(CO)n1-c1ccccc1', 'Clc1ccccc1Oc1cc(C(F)(F)F)ccc1F', 'O=C(CNS(=O)(=O)C1CCCC1)Nc1ccc(F)c(F)c1', 'O=C(Cc1nn(CN2CCCCC2)c2c1CCC2)N1CCC[C@H](Oc2ccccc2Cl)C1', 'Clc1cccc([S@@]Cc2nc(-c3ccccc3)no2)c1', 'CCCNC(=O)CC1CCN(C(=O)c2cccc3[nH]cnc23)CC1', 'Clc1ccc(Br)cc1-n1nnnc1-c1ccccc1', 'Oc1ccc(F)cc1NC(=O)c1cc(F)ccc1O', 'O=[N+]([O-])c1c(F)cc(N2CCN(C(=O)COc3ccccc3)CC2)cc1O', 'Clc1ccccc1Cn1nnc(-c2cccc(F)c2)n1', 'CN(c1ccc(F)cc1)S(=O)(=O)CC(=O)c1ccccc1', 'O=C(CN(c1ccc(Cl)cc1)S(=O)(=O)c1cccc(Cl)c1)N1CCCC1', 'Clc1ccc(-c2ccc(Cl)c(F)c2)cc1Br', 'Clc1ccccc1O/C(=C/c1ccc(Cl)c(Cl)c1)N1CCCCC1', 'Oc1cc(COc2ccc(Cl)cc2)c(Cl)cc1Cl', 'Oc1cccc2cc(OCc3ccc(F)cc3)cnc12', 'Oc1ccccc1CCOc1ccc([N+](=O)[O-])c(Cl)c1', 'Clc1ccc(-c2nn(-c3ccccc3)cc2C(F)(F)F)cc1', 'Clc1ccccc1Oc1cc(Br)c(Cl)cc1Cl', 'O=C(c1ccsc1)c1ccc(COCCO)cc1', 'Clc1ccc(Br)cc1CCCCBr', 'Clc1ccccc1Oc1cnc(CBr)cc1', 'Clc1ccc(-c2cc(Cl)ccc2N2CCCCC2)cn1', 'Clc1ccccc1Oc1cnc(CNc2ccccc2)o1', 'O=C(c1cn[nH]c1NC1CCN(C(=O)COc2ccccc2)CC1)N1CCO[C@@H](C2CC2)C1', 'Oc1cccc(Br)c1Oc1ccc(Br)cc1', 'Clc1ccc([C@@H](NC[C@H](O)c2ccc[nH]2)c2ccccc2)cc1', 'Clc1ccccc1-c1nc(C2CC2)no1', 'Clc1ccccc1Oc1cnc(Cl)c(Cl)c1', 'Clc1ccc(Cc2noc(C(=O)N3CCCCC3)n2)cc1', 'Clc1ccccc1Oc1cnc2scc(-c3ccccc3F)c2c1', 'Clc1ccsc1CNC[C@@H]1OCCN(C(=O)c2cccs2)C1', 'Clc1ccccc1NC1COC1', 'Clc1cc(N2CCc3ccccc3C2)ccc1O', 'Clc1ccc(Br)c(CN2CCc3ccccc3C2)c1', 'Clc1cc(F)c(Cl)cc1C(=O)NNc1ccccc1', 'Oc1ccc([N+](=O)[O-])c(-c2ccc(F)cc2)c1', 'Clc1cccc2ncncc12', 'Clc1cccc2ncncc12', 'Clc1cccc2ncncc12', 'N#CCSc1cccc(OCc2ccccc2)c1', 'Oc1cccc(Oc2ccccc2C(F)(F)F)c1', 'Clc1cccc(Br)c1OCc1ccco1', 'Clc1ccc(-c2n[nH]cc2COc2ccc(F)cc2)cc1', 'Clc1ccc(Br)cc1Cc1ccc(Cl)cc1F', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccc(Cl)cc1', 'O=C(/C=C1CCCCC1)Nc1ccccc1N(Cc1ccccc1)C1CCCCC1', 'Clc1ccccc1Oc1cc([N+](=O)[O-])ccc1O', 'Clc1ccccc1Oc1cc([N+](=O)[O-])ccc1O', 'Clc1ccccc1-c1cscc1-c1cc(=O)[nH]c2ccccc12', 'Clc1cccc(F)c1Cc1nc(CCCc2ccccc2)no1', 'Clc1ccccc1Oc1cc(Br)c(F)cc1Cl', 'Clc1ccccc1-c1ncc(-c2ccccc2Cl)nc1N1CCCC1', 'Oc1cc(F)cc2ncccc12', 'Clc1ccc(-c2nnnn2-c2ccc(F)cc2F)cc1', 'N#Cc1cccc(S(=O)(=O)NC2CC2)c1', 'O=C(CCc1ccc[nH]1)Nc1c(F)cccc1F', 'O=C(CNCc1cccnc1)Nc1cc(Cl)ccc1[N+](=O)[O-]', 'O=C(c1cc(F)ccc1F)N(Cc1ccccc1)Cc1cccs1', 'Oc1ccc(Oc2ccc(-c3noc4cc(F)ccc34)nc2)cc1', 'Clc1ccccc1OCc1ccn[nH]1', 'Clc1ccccc1Cc1cn(CC(=O)[O-])c2ccccc12', 'Oc1cccc(-c2cc[nH]c2)c1CCC(=O)Nc1ccccc1Cl', 'ClCc1ccn(COc2ccccc2)n1', 'O=[N+]([O-])c1ccc2[nH]nc(-c3ccccc3)c2c1', 'Clc1ccc(CO)nc1NCc1ccc(N2CCC2)nc1', 'O=[N+]([O-])c1c([O-])[nH]c(-c2cccc3ccccc23)c1Nc1cccc(Cl)c1', 'Clc1ccc(Br)cc1S(=O)(=O)N1CCOCC1', 'CN(C)/C=C/C(=O)Nc1ccccc1OCc1cccc(Cl)c1', 'Clc1ccc(-c2nnnn2-c2cccc(F)c2F)cc1', 'Clc1ccccc1Cn1cc([N+](=O)[O-])cn1', 'Oc1cccc(NCc2ccc(OCc3cccnc3)cc2)c1Cl', 'CC(C)n1nccc1NC(=O)c1cccc(Cl)c1', 'Clc1ccccc1-c1nc(Cc2ccncc2)no1', 'O=C(c1cccs1)CNc1ccc(Br)cc1', 'O=C(COc1ccc2ccccc2c1)NCC#CCOc1ccc(F)cc1', 'O=C(C1CCN(S(=O)(=O)c2ccc(F)cc2)CC1)N1CCN(C(=O)c2cccc(F)c2)CC1', 'Clc1cccc2ccc(NCc3cc(F)c(F)cc3F)cc12', 'Clc1ccccc1CNc1nc2ccc(C(F)(F)F)cc2n1CC[NH+]1CCC(Cc2ccccc2)CC1', 'Clc1ccc(CO)cc1O', 'ClCc1cc(C#N)ccc1Oc1ccccc1', 'Clc1ccccc1Oc1cc(N2CCOCC2)ccc1O', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n\n', 'Clc1ccccc1CI\n\n\n\n', 'Clc1ccccc1CI\n\n\n', 'Clc1ccccc1CI\n\n', 'Clc1ccccc1CI\n', 'Clc1ccccc1CI', 'Clc1ccccc1CI', 'Clc1ccc(Br)c(Oc2ccc(F)cc2)n1', 'Clc1ccc(-c2cc(Br)c[nH]c2=O)cc1', 'Clc1ccccc1-c1csc(C2CCCC2)n1', 'Clc1cc(Br)cc(Oc2ccccc2Br)c1', 'Clc1ccccc1CNc1n[nH]nc1Cc1ccc(Br)cc1', 'Clc1cc(CCl)nc2ccccc12', 'Clc1cc(Br)cc2cccnc12', 'Clc1cc(Br)cc2cccnc12', 'ClCc1ccnnc1N[C@H](c1ccccc1)c1ccccc1', 'Clc1ccccc1Oc1cc(NC(=O)CC2=CCCCC2)ccc1F', 'O=C(Cc1c(-c2ccc(Cl)cc2)cccc1O)Nc1ccc(C(F)(F)F)cc1', 'Clc1ccccc1Oc1cc(-n2cccn2)nc2cccc(F)c12', 'Clc1cc(CCl)cc(OCc2ccc(Cl)cc2)n1', 'Clc1ccc(Br)cc1S(=O)(=O)Nc1ccc(Cl)c(F)c1', 'Clc1ccc(C(F)(F)F)c(F)c1', 'O=C(CSCCNC(=O)NCc1ccc(F)cc1)Nc1ccc([N+](=O)[O-])cc1', 'O=[N+]([O-])c1c(OCc2[nH]c3ccccc3n2)nc2ccccc2c1N1CCOCC1', 'O=[N+]([O-])c1ccc(OCc2ccc(C(=O)OCc3ccccc3)o2)cc1', 'CNC(=O)[C@H]1CN(CC(=O)Nc2ccc(F)c(F)c2)CCO1', 'Clc1ccccc1Nc1nc2ccccc2cc1CO', 'O=C(Cc1c[nH]c2ccccc12)Nc1ccc(Br)cc1O', 'Clc1cccc2cc(Br)cnc12', 'Clc1cccc2cc(Br)cnc12', 'NS(=O)(=O)Nc1cc(NC(=O)c2ccc3c(c2)CCC3)ccc1Cl', 'Clc1ccc(-c2nnnn2-c2cc(Cl)ccc2Cl)cc1', 'Clc1ccc(-c2noc(-c3cc(F)ccc3Br)n2)cc1', 'Clc1cccc([C@@H](Sc2ccc(Cl)cc2)c2ccc(F)cc2)c1', 'Clc1ccc(Br)cc1C(F)(F)F', 'Clc1ccc([C@@H](O)CCS(=O)(=O)c2ccc(Br)cc2)cc1', 'Clc1ccccc1OCc1csc(Br)c1', 'Oc1ccc([N+](=O)[O-])cc1Cl', 'Clc1ccccc1NS(=O)(=O)Nc1ccc2c(c1)CCC2', 'Clc1ccccc1Oc1cc(NCc2ccccc2Cl)ncn1', 'Oc1cccc(Br)c1COc1cccc(Br)c1', 'Oc1cccc(Oc2ccccc2NCc2ccc(C(F)(F)F)cc2)c1', 'O=C(c1ccsc1)c1c[nH]c2ccccc12', 'CCOc1ccc2[nH]c3ccc(CC(=O)N4CCN(C(C)=O)CC4)cc3c2c1', 'Clc1ccc(-c2nnnn2CCNc2ccc(F)cc2Cl)cc1', 'Clc1ccc(Br)cc1/C=N\\c1ccc(C(C)C)cc1', 'O=C(CNCc1csc(-c2ccccc2)n1)c1ccccc1O', 'O=C(CNCc1c(Cl)cccc1Cl)OCC(=O)Oc1cccc(Cl)c1Cl', 'O=C(COCc1ccccc1[N+](=O)[O-])Nc1c(F)cccc1F', 'O=C(CS(=O)(=O)Cc1cccc(F)c1)Nc1ccccc1OCc1ccccc1', 'Clc1ccccc1Cc1nc(Cn2ccccc2=O)no1', 'Clc1cccc(CCn2ccc(C3CCC3)n2)c1', 'Oc1cc(Cn2nc(-c3ccc(Br)cc3)ccc2=O)ccc1Cl', 'Clc1ccc(-c2cc(F)cc(Br)c2)cc1F', 'Oc1cc(O)cc(N2CCN(C(=O)c3ccc(COc4ccccn4)cc3)CC2)c1', 'O=[N+]([O-])c1ccnc2nc(C3(c4ccc(Br)cc4Cl)CCC3)n(Cc3ccccc3)c(=O)c12', 'C=CCOc1cc([N+](=O)[O-])ccc1OCc1ccc(OC)cc1', 'Clc1ccccc1Oc1cnc(N2CCOCC2)nc1', 'O=[N+]([O-])c1cc(OCc2ccccc2)c(Cl)cc1OCO', 'Clc1ccccc1NS(=O)(=O)NC1CC1', 'OCc1n[nH]cc1Oc1ccc(C(=O)Nc2ccc(Cl)cc2)cc1', 'Clc1ccccc1[C@H](CNC(=O)Nc1ccc(Br)cc1)N1CCOCC1', 'Clc1ccc(-c2cccnc2)nn1', 'Clc1cccc(CCOCCCOc2ccccc2F)n1', 'Clc1ccccc1-c1nc(-c2cccs2)no1', 'Clc1cc(Cl)nnc1-c1ccccc1', 'Clc1cccc(Br)c1Sc1ccccc1', 'O=C(CN(CC1CCCC1)c1ccc(F)cc1)NCC1CC1', 'Oc1ccc(Oc2cc(Cl)ccc2N2CCOCC2)cc1', 'O=[N+]([O-])c1cccc(OCCc2ccco2)c1', 'Oc1cc(O)cc(-c2ncccc2OCc2ccccc2)c1', 'O=C(/C=C/c1cccc(Cl)c1)NCc1cc(Br)ccc1Br', 'Clc1ccccc1Cn1nnc2cc(Cl)c(Cl)cc21', 'O=C(C1CCC1)N1CCN(C(=O)C2CC2)CC1', 'Oc1cccc(Br)c1OCCCc1c(F)cccc1Cl', 'Clc1ccc(-c2cccnc2)cc1N1CCOCC1', 'ClCc1cccc(OCc2cc(-c3ccccc3)n[nH]2)c1', 'Clc1ccccc1CNc1n[nH]c(Cc2ccc(Br)cc2)n1', 'Clc1ccccc1Oc1cnccc1Oc1ccc(OC(F)(F)F)cc1', 'Clc1ccc([C@@H](Br)Cc2cccnc2)cc1', 'O=C(Cc1c(NC2CC2)c[nH]c(=O)c1Cl)NC[C@H]1CCc2ccccc21', 'O=C(c1cnc(N2CCN(CC3CC3)CC2)cn1)N1CCOCC1', 'Clc1ccccc1Oc1cc(OCC(F)(F)F)ccc1O', 'Clc1ccccc1Nc1cnc(Cl)nc1', 'Clc1ccc(Br)cc1-c1cc(=O)oc2ccccc12', 'O=[N+]([O-])c1cc(F)cc(CNc2ccccc2F)c1', 'CN(C)C(=O)c1ccc(NC(=O)c2ccc([N+](=O)[O-])o2)cc1', 'Clc1ccc(-c2cccnc2Cl)nc1', 'Nc1cn[nH]c1-c1ccc(NC(=O)c2ccccc2O)cc1', 'Oc1cccc(-c2csc(COc3ccc(F)cc3)n2)c1', 'Clc1ccccc1NC(=S)NNc1ccc(F)cc1F', 'Clc1cc(Cl)cnc1Cl', 'Clc1ccccc1-c1csc(-c2ccco2)n1', 'Clc1cccc2c(OCCOc3ccccc3Cl)ncnc12', 'Clc1ccc(C(F)(F)F)nc1OCCCCCO', 'NS(=O)(=O)/N=C(\\NCc1ccccc1)c1ccc(-c2ccc(Cl)cc2)cc1', 'Oc1ccccc1-c1c(O)ccc2c1CCCC2', 'O=[N+]([O-])c1cc(C(=O)N2CCCc3ccccc32)ccc1N1CCCC1', 'Clc1ccccc1NS(=O)(=O)N1CCN(Cc2ccc(F)cc2Cl)CC1', 'Clc1ccc(Br)cc1CC(=O)N/N=C/c1ccc(F)cc1', 'Clc1ccccc1C1=CCN(c2ccc(F)cc2)CC1', 'Clc1ccc(Br)c(S(=O)(=O)N2CCC(OCc3ccccc3)CC2)c1', 'O=C(c1c[nH]c2ncn([C@H](CO)c3nc4ccccc4cc3)c12)N1CCN(C(=O)c2ccccc2)CC1', 'Clc1cc(Cl)ncc1Oc1ccc2ccccc2c1', 'Clc1ccc(Br)cc1Sc1cccc(F)c1', 'COc1nc(N2CCCCC2)nc2c1cnn2Cc1ccccc1Cl', 'Clc1ccccc1C(=O)Nc1nnc(-c2ccc(Cl)c(Cl)c2)s1', 'Oc1ccc(F)cc1Nc1cccc(C(=O)N2CCOCC2)c1', 'ClCc1ccn(-c2ccccc2Cl)n1', 'O=C(c1cn(Cc2ccco2)nc1-c1ccco1)c1ccc(Br)cc1', 'ClCc1csc(-c2ccccc2F)n1', 'Oc1ccc(F)c(-c2cccc(F)c2F)c1', 'Oc1ccc(/C=C2/SC(=S)N(c3ccccc3)C2=O)cc1', 'ClC(c1cccs1)c1cc(CC2CCN(C(=O)c3ccc(F)cc3)CC2)nc2ccccc12', 'Clc1ccc(Br)cc1-n1cccn1', 'Clc1ccc(Br)cc1-n1cccn1', 'Clc1ccc(Br)cc1-n1cccn1', 'Clc1cccc(Nc2cc([N+](=O)[O-])ccc2Br)c1', 'Oc1ccccc1Cc1cc(F)ccc1Br', 'C[C@@H](c1nc2ccccc2[nH]1)C(=O)N1CCC(Cc2ccccc2)CC1', 'Cc1cc(=O)oc2cc(NC(=O)/C=C/c3ccccc3)ccc12', 'O=C(CCc1ccncc1)N/N=C/c1ccc(-c2ccccc2F)o1', 'Clc1ccc(CN2C(=O)CNC2=O)cc1', 'Clc1ccccc1Oc1cc(NCCc2nc3ccccc3o2)ccn1', 'Clc1nncn1/c1ccccc1', 'Clc1nncn1-c1ccccc1', 'Clc1nncn1-c1ccccc1', 'Oc1cccc2cc(-c3cccc(Cl)c3)[nH]c12', 'O=C(c1cccn2cnnc12)N(Cc1ccccc1)CC1CCOCC1', 'Clc1cc(Cl)cn2cc(-c3ccccc3)cc12', 'O=C(c1cccc(F)c1F)N1CCC(OCc2cccc(F)c2)CC1', 'Oc1ccc(F)cc1[N+](=O)[O-]', 'C[C@@H](CNC(=O)c1cccc(Br)c1)NC(=O)c1ccccc1', 'Oc1ccccc1/N=N\\c1ccccc1', 'Clc1cccc([S@@]C(C2CC2)C2CC2)c1', 'Clc1ccccc1Cn1nncc1-c1ccccc1Cn1cccn1', 'O=C([C@H]1c2ccccc2C(=O)N(c2c(O)cccc2Cl)C1=O)N1CCCCN1CC1=CCCCC1', 'Clc1ccccc1NCc1csc(N2CCCCC2)n1', 'Clc1ccccc1Nc1nnc(Cc2ccccn2)o1', 'Clc1cc(Br)ccc1Sc1ccc(C=O)cc1', 'Clc1ccc(-c2cccnc2)cc1CN1CCN(c2ccccc2N)CC1', 'Clc1ccccc1-c1ncc[nH]c1=O', 'O=C(CS(=O)(=O)Cc1cccc(Br)c1)N1CCN(c2ccccc2)CC1', 'Clc1ccc(Br)c(Cl)c1CN1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1', 'Clc1ccc(Br)cc1NCCN1CCOCC1', 'Clc1ccccc1OCc1ccnn1C1CC1', 'O=C([O-])c1nn(-c2ccccc2)cc1N1CCN(Cc2ccccc2)CC1', 'Clc1ccc(-c2nonc2Cc2ccc(Cl)cc2)cc1', 'Clc1nnc(-c2ccccc2)o1', 'Oc1cccc(NC2(c3ccccc3)CC2)c1', 'Clc1ccc(-c2nonc2N2CCC(Oc3ccc(F)cc3)CC2)cc1', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n\n', 'Clc1ccc(Br)cc1O\n\n\n', 'Clc1ccc(Br)cc1O\n\n', 'Clc1ccc(Br)cc1O\n', 'Clc1ccc(Br)cc1O', 'Oc1cc(Cn2ncc3cc(Cl)ccc32)ccc1O', 'Clc1ccc(C(F)(F)F)nc1N1CCCCC1', 'Clc1ccc(-c2nnnn2-c2cc(F)ccc2F)cc1', 'Clc1cncc(C2CC2)c1', 'Clc1nnc(COc2ccccc2)n1Cc1ccc2c(c1)OCCO2', 'Oc1ccc(F)c(C(=O)N2CCOCC2)c1', 'Clc1ccccc1OCc1csc2ccc(F)cc12', 'Clc1ccccc1Oc1cc(-c2noc(-c3cccnc3)n2)ncn1', 'Clc1cc(Cl)nc(Nc2ccc(F)cc2)c1', 'Clc1ccc(Br)cc1OS(=O)(=O)Cc1ccccc1', 'N#Cc1c(-c2ccccc2)cc(C(=O)N[C@@H]2CC(=O)N(Cc3ccccc3)C2)n1C1CC1', 'Clc1ccccc1CN(Cn1ncc2ccccc21)C1CC1', 'O=[N+]([O-])c1cc(Br)cc(SCc2ccccc2)c1', 'Clc1ccccc1Nc1cnc(-c2ccccc2)o1', 'Clc1cccc(F)c1-n1cc(Cl)ccc1=O', 'Clc1ccc(CNc2sc3c(n2)CN(Cc2ccc4c(c2)OCO4)CC3)cc1', 'Clc1ccc(-c2cccs2)c2nnc(-c3ccccc3Cl)nc12', 'Clc1cc(Cl)ccc1OS(=O)(=O)c1ccc2c(c1)CCCC2', 'Clc1ccc(-c2csc3[nH]nc(-c4ccccc4)c23)cc1', 'Clc1cccc(F)c1Br\n\n\n\n', 'Clc1cccc(F)c1Br\n\n\n', 'Clc1cccc(F)c1Br\n\n', 'Clc1cccc(F)c1Br\n', 'Clc1cccc(F)c1Br', 'Clc1cccc(F)c1Br', 'Clc1ccccc1CNc1nc(COc2ccccn2)cs1', 'CN[C@H](C)c1ccc(Cl)cc1', 'OCc1nnnn1-c1ccccc1', 'Clc1cccc(/C=N/Nc2c(Cl)cccc2Cl)c1', 'O=C(COc1cc(Cl)c2ccccc2c1O)NNC(=O)COc1ccccc1', 'O=C(COc1cc(Cl)c2ncn(-c3ccccc3[N+](=O)[O-])c2c1)Nc1cc(C(F)(F)F)ccc1Cl', 'Clc1ccc(-c2nonc2CC2CCCC2)cc1', 'Clc1cc(F)cc(Nc2cc(F)ccc2F)c1', 'O=[N+]([O-])c1cccc2ncn(Cc3ccccc3)c12', 'ClCc1ccc(C(=O)N2CCC(Oc3ccc(F)cc3F)CC2)cc1', 'Clc1ccc(Cc2cc[nH]c(=O)c2)cc1', 'O=C(c1cccnc1)N1CCC(Nc2ccccn2)CC1', 'Clc1ccc(Br)cc1S[C@H]1C[C@H](C2CC2)C1', 'O=C(CNc1c(Br)cccc1[N+](=O)[O-])c1ccccc1', 'Clc1ccccc1Oc1cnc2cc(F)ccc2n1', 'Clc1ccccc1Nc1ncc([N+](=O)[O-])cn1', 'O=C(c1ccsc1)c1csc2ccccc12', 'Clc1ccccc1Oc1cc(N2CCCCC2)ncn1', 'Clc1ccc(-c2ccccc2CCSCc2ccc(F)cc2F)cc1', 'Clc1cc(N2CCN(c3ccc(F)cc3)CC2)ncn1', 'Clc1cc(Cl)ccc1N1CCc2cc(Cl)ccc21', 'O=C(CNc1cc2c(cc1Cl)OCO2)c1ccccc1', 'Clc1cccc(Nc2cc(F)cc(Cl)c2F)c1', 'Clc1ccccc1Cc1cnn(CC(=O)N2CC[C@H](Cc3ccc(Cl)cc3)C2)c1', 'Clc1ccc(-c2nnnn2Cc2cccs2)cc1', 'NS(=O)(=O)/N=C\\c1ccc(OCc2ccccc2)cc1', 'Clc1cccc2c(-c3cc([N+](=O)[O-])cc4ccccc34)ccnc12', 'Clc1ccc(Br)cc1OCCS(=O)(=O)c1ccc(F)c(F)c1F', 'Clc1nnnn1c1[nH]ncc1[C@@H](c1ccccc1)N1C(=O)c2ccccc2C1=O', 'ClCc1csc(-c2cc(Cl)ccc2NCc2ccccc2)n1', 'Oc1ccccc1/C=N/NC(=O)c1ccc([N+](=O)[O-])cc1', 'COc1nc2ccc(Cl)cc2cc1C(=O)NCC(=O)NC1CCCC1', 'Clc1ccccc1OCc1csc(-c2ccco2)n1', 'Clc1cc(Br)c2ccccc2c1Cl', 'O=C(Cc1nc(=O)n(-c2ccc(Cl)cc2)[nH]1)NCc1ccc(F)cc1', 'Clc1nnn(C(F)F)c1NCC(=O)N1CCN(c2ccccc2F)CC1', 'Clc1ccc(Br)cc1NCCOCCOCn1ccccc1=O', 'Clc1ccc(Br)cc1NCCS(=O)(=O)c1ccccc1', 'Clc1ccc(-c2ccccc2Cl)nc1O', 'Clc1ccccc1Cc1c(F)cc(Cl)cc1F', 'Oc1ccc(/C=N/Nc2nnnn2-c2ccccc2)cc1', 'Clc1nnnc(Oc2ccccc2)c1N1CCN(c2ccccc2)CC1', 'Oc1ccc([N+](=O)[O-])cc1NCc1cccnc1', 'Clc1nncnc1Nc1ccc(-c2ccccc2)cc1', 'O=C(CN(CCc1cccnc1)C1CC1)Nc1ccccc1C(F)(F)F', 'Clc1ccc(C(F)(F)F)cc1Oc1ccccc1Br', 'Clc1ccccc1OC1(C(=O)[O-])CCN(Cc2ccc(Cl)cc2)CC1', 'Oc1ccc(F)cc1SCC(=O)Nc1ccc2c(c1)COC2', 'Clc1ccc(Br)cc1-c1cnccn1', 'Clc1ccc(Br)cc1-c1cnccn1', 'O=C(/C(Cc1ccccc1)c1cccc(F)c1)c1ccccc1', 'Oc1cccc2nccnc12', 'Clc1ccccc1CNc1nc2ccc(Cl)cc2[nH]1', 'O=C(CSCCO)Nc1ccccc1NCc1cccc(Cl)c1', 'Clc1ccccc1Oc1cc(Cl)c(Br)cc1F', 'Clc1ccccc1-c1cscc1-c1c[nH]c2ccccc12', 'O=[N+]([O-])c1cc(Br)c2ccccc2c1OCc1ccccc1F', 'O=C([C@H](c1c(F)cccc1F)Oc1cccc(C(N)=O)c1)N1CCN(c2ccccc2F)CC1', 'O=C(CNc1ccc(Cl)cc1[N+](=O)[O-])Nc1ccc(F)c(F)c1', 'Clc1ccccc1-c1csc(Cc2cccnc2)n1', 'Clc1ccc(-c2cc(F)ccc2O)cc1N', 'Clc1ccc(Cc2nccc(CCCCCl)n2)cc1', 'O=C([C@H](SC1CCCC1)c1ccccc1)Nc1ccc(Oc2ccccc2)cc1', 'Oc1ccc(/C=N\\CCCCSc2ccccc2)cc1', 'Oc1ccc(F)cc1-n1ccc2cc(F)ccc21', 'O=C(c1c[nH]nc1N(Cc1ccccc1)Cc1ccc(Cl)cc1)C1CC1', 'Clc1ccccc1NS(=O)(=O)c1cc2c(cc1Cl)OCCO2', 'Oc1cccc(CCc2cncc(-c3ccccc3)n2)c1', 'Clc1ccccc1C1(OCCN2CCOCC2)CC1', 'Clc1cccc([S@@]c2nnnn2-c2ccccc2)c1', 'Clc1cc(F)c(N2CCC2)cc1NCC(=O)N1CCOCC1', 'Clc1ccccc1-c1noc(CCCS(=O)(=O)c2ccccc2)n1', 'Clc1ccccc1O/Cn1cc([N+](=O)[O-])cn1', 'Clc1cc(F)c(Nc2ncccc2C(=O)N2CCCCC2)cc1', 'Oc1ccc(F)c(O)c1', 'Oc1ccc(F)cc1-c1nnc(SCCc2ccccc2)n1-c1ccccc1', 'O=C(c1cn[nH]c1)N1CCN(c2ccccc2CNc2nc3ccc(Cl)cc3s2)CC1', 'CCc1cccc(NC(=O)C#CCc2ccccc2)c1', 'Clc1ccc(Br)cc1-c1cn(-c2ccccc2)nn1', 'Clc1ccccc1N1N(CC(=O)NCc2ccc(F)cc2)Nc2ccccc21', 'Oc1cccc(CCNc2c(Cl)cc(Cl)cn2)c1', 'Clc1ccc(COc2ncc3cccc(Br)c3c2)cc1', 'Clc1cccc(Nc2ncnc(Nc3ccccc3)c2[N+](=O)[O-])c1', 'ClCc1cc(F)cnc1Oc1ccccc1', 'O=C(/C=C/c1ccc(Cl)cc1)Nc1ncc[nH]1', 'O=C(COc1cc(Cl)c(Cl)cc1Cl)c1ccc(C(F)(F)F)cc1', 'Oc1ccc(Oc2ccc(NC(=O)Nc3cc(Cl)ccc3O)cc2)cc1F', 'O=[N+]([O-])c1cccc(CNC2CCSCC2)c1', 'O=C(Cc1cc(S(=O)(=O)N2CCOCC2)ccc1O)c1ccccc1', 'O=C(/C=C/c1ccc2c(c1)OCO2)c1cnn(Cc2ccccc2)c1', 'CN(c1ccn(Cc2ccccc2)n1)C1CCCC1', 'CCc1ccc(NC(=O)N2CCN(c3cccc(C)c3)CC2)cc1[N+](=O)[O-]', 'Nc1c(Cl)cccc1NC(=O)[C@@H]1CC(=O)N(CCc2c[nH]c3ccccc23)C1', 'O=C(c1cc(-c2ccccc2)n[nH]1)N1CCN(S(=O)(=O)c2ccccc2Cl)CC1', 'Clc1ccccc1-c1csc(N2CCOCC2)n1', 'O=C(CC1CCN(c2ncccn2)CC1)Nc1ccc(F)c(F)c1', 'O=C(COc1cc(C2CC2)nc2ccccc12)N1CCCC[C@@H]1c1nc2ccccc2[nH]1', 'OCc1nccn1CCCOc1cc(Cl)ccc1Cl', 'Clc1ccc(Br)cc1OCCNC(=O)c1ccc(=O)n(CCC(F)F)c1', 'O=C(Cc1cnc2ccccc2n1)Nc1ccc(N2CCCC2=O)c(Cl)c1', 'Clc1ccccc1-c1nc(CCl)on1', 'Clc1ccccc1Oc1cc([N+](=O)[O-])ccn1', 'O=C(COCc1csc(Br)c1)OCc1ccc(F)cc1', 'COc1ncncc1C(=O)Nc1ccc([N+](=O)[O-])cc1', 'Clc1ccc(-c2nncn2C2CC2)cc1', 'Cc1cc(CC(=O)NCc2nc3ccccc3[nH]2)c2ccccc2n1', 'Oc1cc(O)cc(Oc2ccc(C(=O)NC3CC3)cc2Cl)c1', 'O=C(CSCc1ccccc1)Nc1ccc(S(=O)(=O)C(F)F)cc1', 'O=C(Cc1noc(Cc2ccc(Cl)cc2)n1)NCCc1ccccc1Br', 'ClCc1csc(COc2ccc(Br)cc2)n1', 'O=C(c1c[nH]c(=O)[nH]c1=O)N1CCN(c2ccccc2O)CC1', 'Clc1ccc(-c2ccccc2Cl)nc1C1CC1', 'O=C(c1cn(-c2ccc(F)cc2Cl)nc1NC[C@H]1CN2CCCC[C@@H]2CO1)Nc1ccc(F)cc1', 'Clc1ccc(Br)cc1CN(Cc1ccc(Cl)cc1Cl)C[C@@H](O)C1CC1', 'Clc1ccc(Br)c(-n2cccn2)c1', 'O=C(c1ccsc1)c1nn(-c2ccc(F)cc2)c2ccccc12', 'O=C(/C(CNc1ccccc1)c1ccccc1)c1ccncc1', 'Clc1ccc(-c2nonc2N2CCCCC2)cc1', 'O=C(COc1cc(Br)c(Cl)cc1)NCc1cccc([N+](=O)[O-])c1', 'Clc1cccc(CCOC(F)(F)F)c1', 'O=C(Cc1ccc(Cl)cc1)NC[C@H]1CCCN(c2ncnc3ccccc23)C1', 'Clc1cc(NCCc2ccccc2Cl)ncn1', 'Clc1ccc([C@@H](CCCl)Nc2ccccc2F)cc1', 'ClC[C@H](O)COc1ccccc1', 'COc1nnnc(N(Cc2ccccc2)CC(=O)c2ccc(Cl)cc2)c1O', 'Clc1ccc(CO)c(N2CCN(Cc3ccco3)CC2)c1', 'Clc1ccc(-c2nnn(-c3ccccc3)n2)cc1O', 'O=C(COc1cc([N+](=O)[O-])ccc1OCc1ccc([N+](=O)[O-])cc1)Nc1ccc([N+](=O)[O-])cc1', 'O=C(Cc1nn(C[C@H]2CCCO2)c(=O)c2ccccc12)NCc1ccccc1', 'Clc1ccccc1Nc1cc(Br)cc(F)c1F', 'Clc1nnnn1-c1ccccc1', 'Clc1nnnn1-c1ccccc1', 'O=C(COc1cc([N+](=O)[O-])ccc1[N+](=O)[O-])N1CCCC1', 'Clc1cccc(CN(Cc2nc3ccccc3s2)C2CC2)c1', 'OCc1nc(NCc2ccc(Cl)cc2)ncn1', 'O=C(CSCc1csc(NC(=O)c2ccc(Cl)cc2)n1)Nc1ccccc1', 'O=C(/N=C/c1ccc(F)cc1)c1ccc(NC(=O)c2cccc([N+](=O)[O-])c2)cc1', 'Clc1ccccc1-c1nn(CN2CCc3ccccc32)c2c1CCCC2', 'Clc1ccccc1Oc1cc(Cl)c(N2CCOCC2)c(F)c1', 'ClCc1cn[nH]c1-c1ccccc1', 'CNS(=O)(=O)CC(=O)N1CCN(C(=O)c2cccc(C)c2)CC1', 'Oc1ccc(Oc2cc(-c3ccccc3)ncn2)cc1O', 'Clc1cccc(F)c1[C@@H](c1ccccc1Br)N1CCOCC1', 'Clc1cc(Br)cc(I)c1Oc1ccccc1', 'Oc1ccc([N+](=O)[O-])c(Nc2cccc(Cl)c2)c1', 'Clc1cccc2c(NCc3cccc([N+](=O)[O-])c3)ncnc12', 'Clc1ccc([C@@H](OCc2ccccc2)c2ccccc2)cn1', 'O=C(CNS(=O)(=O)CC1CCC1)Nc1ccc(N2CCOCC2)nc1', 'Clc1cc(Nc2ccc(Cl)c(Cl)c2)c2ccccc2c1Cl', 'Nc1nnc(Cc2cccs2)n1-c1ccccc1', 'Clc1ccc(Br)cc1-c1ccsc1', 'Clc1ccc(Br)cc1-c1ccsc1', 'Clc1cccc(/N=N/c2cccc(Oc3ccccc3)c2)c1', 'Clc1ccc(Br)cc1O[C@@H](c1ccccc1)N1CCCC1', 'Clc1ccc(Br)cc1Cc1ccnc(F)c1', 'Clc1cc(Br)c2nc(-c3ccccc3)[nH]c2c1', 'Nc1[nH]ncc1COc1ccc(Cl)cc1', 'Clc1ccc(Cc2cccs2)nc1', 'NS(=O)(=O)CCCNCc1c(Cl)cccc1Cl', 'Clc1ccc(-c2cccnc2Cl)cn1', 'CCn1nnc(CN2CCN(C(=O)Cc3ccc(F)cc3F)CC2)n1', 'Clc1ccc(Cc2c[nH]nc2-c2ccccc2)cc1', 'O=C(CCCCCn1cncn1)N1CCC[C@@H](COc2ccc(F)cc2)C1', 'Clc1ccccc1Oc1cc(Cc2csc(-c3ccco3)n2)cnc1[O-]', 'CNS(=O)(=O)c1c(C)[nH]c(C(=O)Cc2ccccc2)c1C', 'Clc1cc(Cl)ccc1CSc1ncccn1', 'O=C(Cc1c[nH]c2ccccc12)NC1CCN(c2cccc[nH+]2)CC1', 'Clc1cccc2cc(O)c(F)cc12', 'O=C(Cc1cc(Cc2ccccc2)no1)c1cccc(Cl)c1', 'Clc1ccc(-c2cccnc2Cl)cc1Br', 'Clc1ccccc1OC1(OCc2ccccc2)CCCC1', 'Clc1ccc(COc2nc3[nH]c4ccccc4n3c2NCCCCCCN2CCOCC2)cc1', 'Clc1cc(Cl)c(C(=O)COC(=O)CCCc2ccco2)cc1F', 'Oc1ccccc1-c1c(F)cc(Cl)cc1F', 'C=CCc1cccc(NC(=O)N2CCC(CNC(=O)C(C)C)CC2)c1', 'O=C(/C=C/c1ccc(COc2ccc(Cl)cc2)o1)Nc1cc(Cl)ccc1Cl', 'ClCc1nnc(-c2ccccc2)n1-c1cccc(Cl)c1', 'Clc1ccc(C(F)(F)F)c(NCc2ccc(F)cc2Br)c1', 'Clc1ccccc1Oc1cnc2cc(Br)ccc2n1', 'OC[C@@]1(Cn2cccc2)CN(CCc2[nH+]ccn2Cc2ccc(F)cc2)C1', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccccc1Cl', 'Clc1ccc(-c2nn(-c3ccc(Cl)cc3Cl)cn2)cc1', 'O=C(c1cc(Br)cs1)N1CCC(OCc2ccccc2)CC1', 'O=C(c1c[nH]c2cc([N+](=O)[O-])ccc12)N1CCN(C(=S)NCC2CC2)CC1', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1Oc1ccc(F)cc1', 'O=C(c1cn(Cc2cccs2)nc1-c1ccc(Cl)cc1)N1CCOCC1', 'O=[N+]([O-])c1cc(C2(c3ccccc3Cl)CC2)cn1Cc1ccccc1', 'O=C(/C=C\\Nc1ccccc1[N+](=O)[O-])c1ccccc1F', 'Clc1ccccc1-c1csc(Cl)n1', 'Clc1ccccc1-c1csc(Cl)n1', 'Clc1ccccc1Nc1ncnc2c1CCCC2', 'Clc1ccc(C2CCN(C(=O)C(F)(F)F)CC2)cc1', 'O=C(COc1cc(Cl)ccc1C(=O)[O-])N1CCc2ccccc2C1', 'Clc1ccccc1-c1csc(CCl)n1', 'Clc1ccccc1Oc1cnc(N2CCCCC2)cc1Cl', 'Oc1ccc(F)cc1CNc1cc(Br)ccc1F', 'O=C(CCNC(=O)Cn1c(N(C)Cc2ccccc2)nn(-c2ccccc2)c1=O)N1CCOCC1', 'ClCc1nnc(-c2ccco2)n1-c1cccc2ccccc12', 'O=[N+]([O-])c1ccncc1OCCN1C(=O)c2ccccc2C1=O', 'Oc1cccc(-c2cc(CCOc3cccc4ccccc34)no2)c1', 'Clc1ccc(C(=O)Oc2ccccc2C(=O)[O-])cc1', 'O=C(c1c[nH]c2nc[nH]c(=O)c12)N(Cc1ccccc1)c1ccccn1', 'Clc1ccc(Br)c(Cl)c1C1CCN(Cc2cn3cc(-c4ccccc4)cc3nc2)CC1', 'Clc1cccc([S@@]CCCCC(=O)Nc2ccc3c(c2)OCCC3)c1', 'O=C(c1cc(Cl)cnc1Cl)N1CCC(Oc2ccccc2)CC1', 'O=C(c1c[nH]c2cc(Cl)ccc21)N1CC=C(c2ccccc2)CC1', 'CNC(=O)NCC(=O)Nc1ccc(N(Cc2ccccc2)C(C)C)nc1', 'O=C(c1ccsc1)N1CCC(Oc2ccc(F)cc2)CC1', 'Clc1ccc(CC[C@@H](Cl)c2cccc(Cl)c2Cl)cc1', 'O=C(CNCc1ccc([N+](=O)[O-])cc1)Nc1c(F)cccc1F', 'CCS(=O)(=O)CCN(C)C(=O)CCc1ccccc1Cl', 'NS(=O)(=O)CC(=O)Nc1ccc(C(=O)NCc2ccco2)cc1', 'Oc1cc(OC2=NN(c3ncccn3)C[C@@H]2c2ccccc2)ccc1OCCOc1ccccc1F', 'Clc1ccccc1NS(=O)(=O)NCc1cccc([N+](=O)[O-])c1', 'Clc1ccccc1Oc1cnc(/N=C/c2ccccc2Cl)cc1', 'Clc1ccccc1NS(=O)(=O)c1c[nH]c2cc(Cl)ccc12', 'Oc1cccc(SCc2nc(-c3ccccc3)no2)c1', 'Oc1cccc(Br)c1O\n\n', 'Oc1cccc(Br)c1O\n', 'Oc1cccc(Br)c1O', 'O=C(Cc1c(Cl)cccc1Cl)Nc1c(Cl)ccc2cccnc12', 'Clc1ccc(COCc2cccc(C(Br)(F)F)c2)cc1', 'Clc1ccc(Br)cc1-c1cncnc1', 'Clc1ccc(Br)cc1-c1cncnc1', 'O=C(CSCNOCc1ccccc1)N1CCCC1', 'Clc1cc(Cl)nnc1Oc1ccc(F)cc1NCc1ccccc1', 'Clc1ccc(Br)c(Sc2ccc(OCc3ccccc3)cn2)c1', 'Oc1ccc([N+](=O)[O-])c(COc2ccc(C(F)(F)F)cc2)c1', 'O=[N+]([O-])c1ccnc2ccc(F)cc12', 'NS(=O)(=O)CCNC(=O)c1cccc(NC(=O)c2ccco2)c1', 'Clc1ccccc1Nc1nncc(-c2ccccc2Br)n1', 'OC[C@@H](NS(=O)(=O)c1ccc(Cl)cc1Cl)c1ccc(Cl)cc1', 'Clc1cc(Br)cc(CN2CCO[C@@H](c3ccc(C(F)(F)F)cc3)C2)c1', 'O=C(Cc1cn2cccnc2n1)NCc1ccnc2ccccc12', 'Clc1cccc2c(N3CCOCC3)nc3ccccc3c12', 'Oc1cccc(C(=O)Nc2nc(-c3ccc(F)cc3F)cs2)c1', 'Oc1cccc(-n2c(SCC(=O)N3CCCC3)nc3ccccc32)c1', 'CCC(CC)C(=O)N[C@H](C)c1ccc(OC)cc1', 'Oc1cc(COc2ccccc2C=O)c2ccccc2c1O', 'Clc1ccccc1Oc1cc(OCc2ccc(Cl)cc2)cc2c1OCO2', 'Clc1ccc(CC[C@H]2CCCN(c3ncccn3)C2)cc1', 'Oc1cc(C(F)(F)F)ccc1NCc1ccc(N2CCCC2)cc1', 'O=C(/N=c1/[nH]c[nH+]c2ccccc12)c1ccc2c(c1)C(=O)N(C1CCCC1)C2', 'N=Cc1ccc(O[C@H](C)C(=O)N2c3ccccc3C[C@H]2c2ccccc2)cc1', 'N#Cc1ccc(CN2CCN(C(=O)COc3ccccc3F)CC2)nc1', 'Oc1ccc(/C=N/c2cccc(Br)c2)cc1', 'Clc1ccc(-c2n[nH]c(NNc3ccccc3Cl)n2)cc1', 'N#Cc1nccn1/c1ccc(OCc2ccccc2)cc1', 'N=Cc1nc(-c2ccccc2)nc2ccccc12', 'Oc1cc(F)c(Nc2ccc(Br)cc2)cc1Cl', 'O=C(c1cn2ccsc2c1NC[C@H](O)c1ccc(Br)cc1)N1CCC(O)CC1', 'NS(=O)(=O)/N=C(/NCc1ccccc1)NC1CCN(c2ccccc2)CC1', 'Clc1ccc(C(F)(F)F)cc1OC(=O)c1ccc(Cl)c(Cl)c1', 'O=C(CNc1cc(F)c(F)cc1[N+](=O)[O-])Nc1ccccc1F', 'Oc1cc(-n2nc3ccccc3c2[N+](=O)[O-])cc(OCc2ccccc2)c1', 'Nc1nnc(Nc2cccc(Oc3ccccc3)c2)s1', 'Clc1ccc(Br)c(O)c1', 'Clc1cc(N2CCCc3ccccc32)ncn1', 'O=C(c1ccc(=O)n(Cc2ccccc2)c1)N1CCC(OC[C@H]2CCCO2)CC1', 'Clc1ccccc1NC(=O)/C(=C/Cc1cccs1)c1ccccc1', 'Clc1cccc([S@@]Cc2cccc3c2OCO3)c1', 'Clc1ccc(COC[C@@H](O)c2ccccc2)cc1', 'Clc1ccc(Br)c(Cn2ccc3ccc(O)cc32)c1', 'O=C(CSCc1ccc(O)cc1)Nc1cccc(OC(F)(F)F)c1', 'Clc1cc(/C=N/c2cccnc2)ccc1Cl', 'Oc1ccc(/C=C/Nc2ccc(S(=O)(=O)N3CCOCC3)cc2)cc1', 'Clc1cccc(Br)c1O', 'O=C(COc1cc(Cl)ccc1I)Nc1nc2c(s1)CO[C@@H](c1ccccc1)C2', 'O=C([C@H](Nc1ccc(F)cc1)C(=O)N1CCN(c2ccccc2)CC1)N1CCCC1', 'O=C(c1ccsc1)c1ccccc1COC1CCCCC1', 'Clc1ccccc1Oc1cnc2nnc(-c3ccc(C(F)(F)F)cc3)n2c1', 'Clc1cccc2c(-c3cccc(Br)c3)cnn12', 'Clc1cccc([S@@]CCc2ccc(CO)cc2)n1', 'Clc1ccccc1CN(C(=O)/C=C/c1ccc(Cl)cc1Cl)c1ccccc1', 'Clc1ccc(Cc2nccnc2C(=O)NNC(=O)C2CC2)cc1', 'ClCc1csc(C(=O)N2CCN(Cc3ccccc3Cl)CC2)c1', 'ClC(F)(F)c1ccccc1OCc1nccn1Cc1ccccc1', 'O=C(c1c[nH]c2ncn(CC(=O)NCCc3ccccc3)c12)N1CCOCC1', 'Clc1ccc(Br)cc1NCCn1cccn1', 'Clc1ccccc1Oc1cc([N+](=O)[O-])c2ccccc2n1', 'Clc1cc(Br)cnc1Oc1cccc(Br)c1', 'Clc1ccccc1Oc1cnc(Cl)cn1', 'Clc1ccccc1Oc1cncc(Cl)n1', 'Clc1ccccc1N1N(CCc2cccc(Cl)c2)C=C1COc1ccccc1F', 'Clc1ccccc1Nc1cc([N+](=O)[O-])c(Cl)cc1Cl', 'Clc1ccccc1[C@H](NC1CC1)c1sccc1Br', 'Clc1ccccc1Nc1nnnn1-c1ccccc1F', 'NS(=O)(=O)Cc1cccc(NC(=O)c2cnn(-c3ccccc3)c2)c1', 'O=C(c1ccc[nH]c1=O)N(CC1=CCCCC1)C1CCCCC1', 'Clc1ccc(-c2ccnc3ncccc23)cc1F', 'O=C(CSc1nc2ccccc2c(=O)n1Cc1ccccc1)NNC(=O)c1ccccc1F', 'Clc1ccc(Cc2ccco2)nc1', 'Nc1nnnn1CC(=O)NCc1ccccc1OC(F)F', 'Clc1ccccc1-c1noc(Cc2cccs2)n1', 'Clc1ccccc1Oc1cnc(Oc2ccc(Br)cc2F)cn1', 'O=C(CNS(=O)(=O)Cc1ccccc1)Nc1ccc(F)c(F)c1F', 'Oc1ccc(F)cc1-c1cccc(F)c1O', 'Oc1ccc([C@H](/C=C/c2ccc(Br)cc2)c2ccccc2)cc1', 'Oc1cc(-n2cc(Br)cn2)ccc1NCc1ccc(F)cc1', 'Clc1cc(Cl)ccc1I', 'Clc1ccc(Br)cc1OCc1cc(Cl)c2c(c1)OCO2', 'Oc1cc(O)ccc1CN1CCN(S(=O)(=O)c2cccc(Cl)c2)CC1', 'CNc1cccc(CNC(=O)Nc2ccc(F)c(Cl)c2)c1O', 'CNC(=O)Nc1cccc(C(=O)N2CCCc3cc(C)ccc32)c1', 'Clc1ccccc1Oc1cc(OC2CCCCC2)c[nH]c1=O', 'O=C([C@H](c1c(F)cccc1F)N1CCN(c2ccccc2)CC1)N1CCCC1', 'O=C(COCCN=C(c1ccccc1)c1ccccc1)Nc1nccs1', 'O[C@@H](c1ccc(Cl)cc1Cl)c1ccccc1F', 'O=C(COc1cc([N+](=O)[O-])ccc1F)NCC(=O)N1CCOCC1', 'O=C(c1cn[nH]c1-c1ccccc1)N1CCc2cc(S(=O)(=O)N3CCCC3)ccc21', 'O=C(Cc1c(F)cccc1F)c1ccc2c(c1)OCO2', 'Oc1ccc(/C=C2/SC(=S)N(c3cccc4ccccc34)C2=O)cc1', 'COc1ncnc(NCc2ccccc2OCC2CC2)c1C', 'C=C1c2cc3ccccc3cc2C(=O)N1c1ccc(C)cc1', 'Clc1cc(CNCc2ccc(C3CC3)cc2)n[nH]1', 'O=C(/C=C/c1c(F)cccc1Cl)Nc1ccc([N+](=O)[O-])cc1', 'O=C(c1cccs1)c1cnn(-c2ccc(F)cc2)c1', 'O=C(/C=C\\Nc1nccs1)NCCc1ccccc1', 'O=C(/C(CO)=O)c1cc(F)ccc1Oc1ccccc1', 'O=[N+]([O-])c1c(F)cccc1Cl', 'Clc1ccccc1-c1ncc(Cl)s1', 'CC(C)NC(=O)C(=O)Nc1cccc(SC(C)(C)C)c1', 'O=C(CCc1ccc([N+](=O)[O-])cc1)N1CCN(C(=O)c2sc3ccc(F)cc3c2C)CC1', 'COc1nnn(-c2ccc(OC)cc2)c1C(C)C', 'Clc1cc(/N=C\\c2ccc(OCc3ccccc3)cc2)nc(Cl)c1', 'O=C(CNS(=O)(=O)c1ccccc1)NNC(=O)c1cc(-c2ccccc2)on1', 'ClC(=S)O/C(=C/c1ccc(N2CCCC2)cc1)c1ccc(F)cc1', 'Oc1ccc(/C=C(\\Cl)/C=C(Nc2ccc(Cl)cc2)N2CCOCC2)cc1', 'Oc1cc(OCC(F)F)ccc1Oc1cccc(Cl)c1', 'Clc1ccccc1Oc1cc(NCc2cccnc2)ncn1', 'Clc1cccc2cc(CNS(=O)(=O)c3ccccc3Cl)[nH]c12', 'Clc1ccccc1O/N=C\\c1cccc(Cl)c1', 'O=C(COc1cc(Nc2ccccc2)ccc1O)NNN1CCCCC1', 'Clc1ccccc1OCc1ccn(-c2cccc(F)c2Cl)n1', 'O=C(c1cnccn1)N(Cc1cccc(Cl)c1)c1nc2ccc(F)cc2s1', 'Clc1ccccc1OCc1csc(Cl)c1', 'O=C(Cc1nnnn1-c1ccc(Br)cc1)Nc1ccc2c(c1)OCCO2', 'Clc1ccc(C2(OCc3ccc(-n4cnnc4)cc3)CCC2)cc1', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n\n', 'Clc1cccc2[nH]ncc12\n\n\n', 'Clc1cccc2[nH]ncc12\n\n', 'Clc1cccc2[nH]ncc12\n', 'Clc1cccc2[nH]ncc12', 'Clc1cccc2[nH]ncc12', 'Clc1cccc2[nH]ncc12', 'Clc1cccc2[nH]ncc12', 'O=C(c1cncc(Cn2cncn2)n1)Nc1ccccc1Cl', 'Clc1ccc(COc2nc(Br)ccn2)cc1', 'Clc1ccc(-c2ccccc2)nc1CNC[C@@H](O)c1ccccc1F', 'Clc1cccc(F)c1C(=O)Nc1ccc(-c2cscn2)cc1', 'Oc1ccc(/C=N/NC(=O)c2ccc(-c3ccccn3)s2)cc1', 'Clc1ccc(-c2cc(Br)c3c(c2)OCCO3)cc1', 'Clc1cc(F)c(Oc2cncc(C(F)(F)F)c2)cc1', 'Clc1nncc(-c2ccc(F)cc2)n1', 'Clc1nncc(-c2ccc(F)cc2)n1', 'Clc1ccc(Br)cc1OCCCOCC(F)(F)F', 'O=C(c1cc(Br)c2c(c1)OCO2)N1CCC(Oc2ccccc2)CC1', 'Clc1cc(Br)cnc1N1CCOCC1', 'ClCc1nonc1N/N=C/c1ccc(-c2ccccc2)cc1', 'Clc1cccc2cc(F)ncc12', 'Clc1cccc2cc(F)ncc12', 'O=C(/C=C/c1c(N)n(CCCO)c1=O)c1ccccc1', 'O=C(/N=c1\\occcc1OCCCCc1ccccc1Cl)c1cnn(-c2ccccc2)c1', 'NS(=O)(=O)/C=C\\c1ccc(-n2cccc(C(=O)Nc3ccccc3)c2=O)cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])c(F)cc(C(=O)NCCc2ccccc2)c1', 'Clc1cc(CNc2ccc(N3CCOCC3)cc2)nc(-c2ccncc2)n1', 'Oc1ccc(Oc2ccc(C(=O)[O-])cc2)cc1', 'Oc1ccc(/C=N\\Nc2nc(Cl)ccn2)cc1', 'O=C(/C=C/c1cnccn1)Nc1cccc(Oc2ccccc2)c1', 'O=C(/C=C/O)c1ccc(Cl)cc1', 'Clc1ccccc1-c1n[nH]c2c1CCCC2', 'O=C(CNC(=O)c1ccc([N+](=O)[O-])cc1)NNC(=O)c1ccccc1[N+](=O)[O-]', 'Oc1ccc(F)cc1NC(=O)c1ccc([N+](=O)[O-])cc1O', 'O=C(c1ccc(C(F)(F)F)cc1)N1CCC(Oc2ccc(-c3ccccc3)s2)CC1', 'Clc1ccc(Br)cc1SCCC(F)(F)F', 'Clc1ccccc1Nc1ccc2nnc(-c3c(F)cccc3Cl)n2c1', 'OCc1ncc(CCOCc2ccccc2)s1', 'O=[N+]([O-])c1ccc(F)cc1NC(=O)COC(=O)c1ccc2c(c1)OCO2', 'Clc1cccc(SCc2nnnn2Cc2ccccc2)c1', 'O=C(CS(=O)(=O)Cc1cccc(Cl)c1)Nc1c(F)cccc1F', 'Clc1ccccc1C(C)CCOc1c(Cl)cccc1Cl', 'ClCc1cnn(-c2ccccc2)c1N', 'NS(=O)(=O)/N=C/c1ccc(-c2ccc(Cl)cc2)cc1', 'O=C(c1cccc(Br)c1)N1CCN(c2nccc([N+](=O)[O-])c2)CC1', 'Clc1cc(F)c([N+](=O)[O-])cc1OCc1ccccc1[N+](=O)[O-]', 'Clc1ccccc1O/Cn1nnnc1Cc1cccs1', 'Clc1ccsc1CN(Cc1cccc(F)c1)c1ccccc1', 'Clc1cc(Cl)ccc1Cn1cc(C(F)(F)F)ccc1=O', 'Oc1cc(O)cc(C(=O)N2CCN(S(=O)(=O)Cc3ccc4c(c3)CCCC4)CC2)c1', 'Clc1ccc([C@@H](NCc2ccccn2)Cc2cccnc2)cc1', 'Clc1ccccc1NS(=O)(=O)c1ccc(C(=O)[O-])cc1', 'Clc1ccccc1NS(=O)(=O)c1ccc(C(=O)[O-])cc1', 'O=C(CNCc1nc(-c2ccccn2)no1)N1CCN(C[C@@H]2CCCO2)CC1', 'O=[N+]([O-])c1c(Br)sc(Br)c1/C=N/OCCn1ccc2ccccc21', 'Oc1ccc(F)c(F)c1F', 'Clc1ccccc1Nc1cn(CCc2ccccc2)nc1CSc1ccccc1', 'Clc1cc(Br)cc2cnccc12', 'Clc1ccc(Br)cc1CCn1cccn1', 'Clc1ccccc1Oc1cc(SC(F)(F)F)ccc1Cl', 'O=[N+]([O-])c1c(Br)c(F)cc(OCC(=O)N2CCC[C@@H](Cc3ccccc3)C2)c1', 'Clc1cccc([S@@]CCOc2ccc(Br)cn2)c1', 'O=C(/C=C\\C=C\\c1ccccc1)Nc1ccc2c(c1)OCO2', 'Clc1cc(CNC2CC2)nc(-c2ccc(F)c(F)c2)n1', 'Clc1ccc(Br)cc1Sc1nc2ccccc2n1Cc1ccccc1Cl', 'Clc1ccc(Br)cc1OS(=O)(=O)c1cccc(Br)c1', 'Clc1ccccc1CCc1cccc(C(=O)N2CCC[C@@H]2c2cccc(F)c2)c1', 'Clc1ccc(-c2ccccc2Cl)c(-n2cnnn2)c1', 'Clc1ccccc1-c1nnnn1Cc1ccc(F)cc1F', 'O=[N+]([O-])c1cc(S(=O)(=O)Nc2ccc(Cl)c(Cl)c2)cc(Cl)c1Cl', 'O=C(/C=C/c1cccs1)N1CCN(c2ccc(Cl)cc2Cl)CC1', 'Clc1ccc(-c2cc(Br)ccc2N2CCOCC2)cn1', 'CNC(=O)c1ccc(NC(=O)c2sc(C(=O)N3CCCCC3)cc2C)cc1', 'O=C(COc1cc(Cl)ccc1F)NNC(=O)CCC(=O)c1ccc(F)cc1F', 'Clc1nnnn1c1ccc(OCC(N)=O)cc1', 'Oc1ccc(F)c(F)c1CNc1cccc2c1CCCC2', 'O=C(C1CC1)N/C=C/c1ccnc2ccccc12', 'O=C(CN(C[C@@H]1CCCO1)c1ccc(Cl)cc1)N1CCOCC1', 'O=[N+]([O-])c1c(Cl)nc(N2CCCC2)nc1Oc1ccc(Cl)cc1', 'Clc1ccc(Br)c(CCl)c1', 'Clc1ccc(C(N)=Nc2ccc([N+](=O)[O-])cc2)cc1', 'O=C(Cc1n[nH]c2c(Cl)cccc12)Nc1ccccc1F', 'Clc1cc(CN(Cc2ccccn2)C2CC2)ccn1', 'Clc1ccc(-c2cc[nH]n2)cc1CCNC(=O)c1ccccc1', 'Oc1ccc(/C=C(/Cl)c2ccccc2[N+](=O)[O-])cc1', 'Clc1ccc(Br)cc1NCCCc1nnc2ccccn12', 'Oc1cc(-n2nnc(-c3ccc(Cl)cc3)n2)ccc1N1CCCCC1', 'OCS(=O)(=O)c1ccc(F)cc1Oc1ccc(C(F)(F)F)cc1', 'Clc1ccc(Br)cc1-c1ccc(CO)o1', 'Oc1ccc(F)c(-c2csc(Cc3ccc(F)cc3)n2)c1', 'Clc1cc(CO)ccc1C(F)(F)F', 'Clc1cc(CCl)ccc1Br', 'Clc1cc(CCl)ccc1Br', 'O=C(c1cn2ccccc2n1)N1CCN(Cc2ccccc2F)[C@H](C(=O)c2ccc(Cl)cc2)C1', 'Clc1cc(F)cc(Oc2ccncc2F)c1', 'Clc1cccc(Br)c1ONCc1ccc[nH]1', 'Oc1cccc(-c2cn[nH]c2)c1', 'Clc1ccc(-c2nonc2NC(=O)Cc2c(Cl)cccc2Cl)cc1', 'Clc1cc(Nc2ccccc2Cl)nc(-c2ccccn2)n1', 'COc1nc2nc(CCc3ccc(F)cc3)nn2c(=O)n1Cc1ccccc1', 'OCS(=O)(=O)c1ccccc1F', 'O=C(Cc1coc(-c2ccc(Cl)cc2)n1)N1CCCC[C@@H]1c1ccccc1', 'Oc1cc(F)ccc1Br', 'Clc1ccc(-c2cccs2)nc1OC1CCN(C(=O)c2ccccc2)CC1', 'Clc1ccc(-c2cccs2)nc1N1CCc2ccccc21', 'Nc1c(Oc2ccc(F)cc2)nnn1-c1ccc(F)cc1', 'Oc1cccc(Br)c1CNCc1ccc(Br)c(F)c1', 'CC(=O)N1CCC(CCC(=O)NNC(=O)c2cn(C)nc2-c2ccccc2)CC1', 'OCSc1ccc2cc(Cl)ccc2c1', 'O=C(/C=C/c1c(F)cccc1Cl)Oc1ccccc1[N+](=O)[O-]', 'O=C(/C(Cc1ccccc1)c1ccc(F)cc1)c1c(F)cccc1F', 'Clc1ccccc1Oc1cc(Cl)c([N+](=O)[O-])cc1[N+](=O)[O-]', 'Clc1ccc(C2SCCS2)cc1', 'Clc1ccc(C2SCCS2)cc1', 'O=C(COc1cc(Br)cnc1Cl)N(Cc1ccc(F)cc1)C1CC1', 'O=C(c1cnn(Cc2ccccc2C#N)c1)N1CCCCC1', 'Clc1ccccc1OCCCSc1nnc2n1CCCC2', 'O=C(c1cc(F)ccc1C(=O)c1ccc(Cl)s1)N1CCCCC1', 'Clc1ccc(-c2nnnn2-c2ccccc2Br)cc1F', 'Clc1ccc(Br)cc1/N=C/c1ccc(F)cc1', 'Clc1ccccc1-c1csc(S(=O)(=O)Cc2ccccc2)n1', 'Oc1ccc([C@H](NC(=O)c2cccc(Cl)c2)c2ccccc2)cc1', 'CN(c1cccc(N)c1)S(=O)(=O)c1cccc([N+](=O)[O-])c1', 'C[C@@H](NCc1ccc(Cl)cc1Cl)c1ccc(Cl)cc1', 'Clc1cc(F)c(N/N=C/c2ccccc2O)cc1', 'Oc1ccccc1OS(=O)(=O)c1ccccc1NC(=O)CC1CCCC1', 'O=C(Cc1noc2nc(-c3ccccc3)oc12)Nc1ccc(F)cc1', 'O=[N+]([O-])c1ccn(Cn2c(=O)c3ccccc3n2Cc2ccccc2)c1', 'Clc1ccc(-c2n[nH]cc2-c2ccccc2F)cc1', 'Clc1ccc(-c2n[nH]cc2-c2ccccc2F)cc1', 'Clc1nnnn1CCOc1ccc(Br)cc1', 'Clc1cc(COCOc2ccc(Cl)cc2)ncn1', 'Oc1ccc(F)cc1-n1cccc1C(=O)NCc1ccccc1Br', 'O=C(CNc1cccc(OC(F)(F)F)c1)N1CCN(S(=O)(=O)c2ccccc2)CC1', 'Clc1cc(F)cc(NNC(=O)COc2cccc(Br)c2)c1', 'Clc1ccccc1Oc1cc(F)c(F)c(F)c1F', 'Clc1ccc(-c2noc(-c3cccc(C(F)(F)F)c3)n2)c(Cl)c1', 'Clc1ccccc1-c1cscc1F\n\n\n\n\n\n\n\n\n', 'Clc1ccccc1-c1cscc1F\n\n\n\n\n\n\n\n', 'Clc1ccccc1-c1cscc1F\n\n\n\n\n\n\n', 'Clc1ccccc1-c1cscc1F\n\n\n\n\n\n', 'Clc1ccccc1-c1cscc1F\n\n\n\n\n', 'Clc1ccccc1-c1cscc1F\n\n\n\n', 'Clc1ccccc1-c1cscc1F\n\n\n', 'Clc1ccccc1-c1cscc1F\n\n', 'Clc1ccccc1-c1cscc1F\n', 'Clc1ccccc1-c1cscc1F', 'Clc1ccccc1-c1cscc1F', 'Clc1ccc(C2SCCCS2)cc1', 'Clc1cc(Br)cnc1COc1ccc(Cl)cc1Cl', 'Clc1ccccc1Oc1cnc(Br)cn1', 'Clc1ccc(-c2cccs2)o1', 'Clc1ccc(C2(F)CC2)cc1', 'CNC(=O)Cc1cc(=O)oc2ccc(Cl)cc12', 'Clc1ccc(-c2nnnn2-c2ccccc2)c2c1CCC2', 'ClCc1ncnn1-c1ccccc1', 'Clc1cccc(NCCn2cc([N+](=O)[O-])cn2)c1', 'Clc1cccc(NCC(=O)Nc2ccncc2C(F)(F)F)c1', 'Clc1cccc2[nH]nc(N/N=C/c3ccccc3)c12', 'O=C(c1cccnc1Cl)N1CCC(CCNc2cccc(Cl)c2)CC1', 'O=C(CNS(=O)(=O)c1cccs1)NC1CC1', 'Nc1nncn1-c1ccccc1', 'Clc1ccc(-c2cccnc2S(=O)(=O)N2CCCCC2)cc1', 'Clc1ccc(Br)cc1-c1ccc2c(c1)nc(Cn1cccn1)n2C1CC1', 'Oc1cc(-n2nnnc2-c2ccccc2F)cc(-c2ccsc2)n1', 'Nc1nn(-c2ccc(N3CCCCC3)cc2)c2c1CCC2', 'O=C([O-])CSc1ccccc1C(=O)Nc1ccc(OC2CCCC2)cc1', 'Oc1cccc(Br)c1Oc1ccc(F)cc1Cl', 'Oc1ccc(Oc2cc([N+](=O)[O-])ccc2N2CCOCC2)cc1', 'O=C(c1ccsc1)N(CCO)Cc1cccnc1', 'Clc1cc(Cl)c(-n2cc(I)c3ccccc32)nc1', 'Clc1ccccc1Oc1cncc(-c2ccc(F)cc2Cl)n1', 'Oc1ccccc1C1=C(N/N=C/c2ccccc2)C=C1c1cc[nH]c1', 'OCc1nc2ccc(F)cc2n1CCN1CCOCC1', 'O=C([C@H](CSc1ccccc1)NCc1ccccc1)c1ccccc1', 'Clc1ccc(-c2cc(NCc3cccs3)no2)cc1', 'ClC(=S)NCC(=O)Nc1ccccc1F', 'Clc1ccccc1-c1cscc1Br', 'Oc1cc(-n2cccc2)cc(Cn2ccc3ccc(Cl)cc32)c1', 'Clc1ccc(Br)cc1-c1cnn(-c2ccc(Cl)cc2Cl)c1', 'O=[N+]([O-])c1cc(Cl)c(NCc2ccncc2)cc1F', 'O=[N+]([O-])c1ccnc2ccc(Br)cc12', 'O[C@@H]1c2ccccc2C(=O)N1CCOc1ccc(Br)cc1', 'Oc1cc(Br)cc(CCc2nnc(-c3ccc(Cl)cc3)o2)c1', 'Clc1ccc(-c2csc3s/Cc4ccccc4c23)cc1', 'Clc1ccc(-c2cccs2)nn1', 'Clc1ccc(-c2cccs2)nn1', 'Clc1ccc(-c2cccs2)nn1', 'O=C(c1cc(Cl)cc2c1OCCO2)c1ccccc1', 'Clc1ccccc1[C@H](Cl)c1ccc(F)cc1', 'Clc1ccccc1NCc1cncn1Cc1cccs1', 'O=C(Cc1cnccn1)Nc1cc(Cl)c(Cl)cc1Cl', 'Nc1[nH]ncc1CCC(=O)Nc1ccn(-c2ccccc2)n1', 'Oc1ccc([N+](=O)[O-])c(Cl)c1Oc1cccc2ccccc12', 'Clc1ccc(Br)cc1N(C[C@H]1CCCO1)c1ccccc1', 'O=C([O-])/C(=C/c1ccccc1)c1ccc(O)cc1', 'Clc1ccccc1Cn1c2ccccc2c2ncnn21', 'Clc1cccc(Br)c1Br\n\n\n\n\n\n\n\n', 'Clc1cccc(Br)c1Br\n\n\n\n\n\n\n', 'Clc1cccc(Br)c1Br\n\n\n\n\n\n', 'Clc1cccc(Br)c1Br\n\n\n\n\n', 'Clc1cccc(Br)c1Br\n\n\n\n', 'Clc1cccc(Br)c1Br\n\n\n', 'Clc1cccc(Br)c1Br\n\n', 'Clc1cccc(Br)c1Br\n', 'Clc1cccc(Br)c1Br', 'Clc1cccc(Br)c1Br', 'Clc1ccsc1-c1ccncc1', 'NS(=O)(=O)Cc1cnn(-c2ccccc2)c1', 'Nc1nncn1CCc1ccccc1', 'Clc1cccc2cc(CNc3nccs3)ccc12', 'O=C(CNCc1noc2ncc(-c3ccccc3)cc12)Nc1ccccc1F', 'CNC(=O)NNc1ccc(C(=O)NC(C)C)c(Cl)c1', 'O=C(c1cnc(OCCc2ccccc2)cc1)N1CCC[C@@H](OCc2ccccc2F)C1', 'Clc1ccccc1NC1CSC1', 'Oc1cc(COc2ncccc2Cl)ccc1F', 'Clc1ccc(Br)cc1OCC(=O)NC(C1CC1)C1CC1', 'Clc1ccc(-c2n[nH]c3[nH]c4ccccc4c23)cc1', 'Clc1ccc([C@@H](Br)COc2ccccc2)cc1', 'Clc1ccc(-c2cc(Oc3ccccc3)nc(SC3CC3)n2)cc1', 'O=[N+]([O-])c1c(F)cccc1OCCN1CCN(C(=O)c2cccs2)CC1', 'Clc1ccc(-c2cccs2)nc1SCc1ccccc1Cl', 'O=C(c1cc(Cl)cc(OC(F)F)c1)N1CCOCC1', 'O=C(c1ccsc1)[C@@H]1CN(Cc2cccs2)CCO1', 'O=C(CNc1c(F)cccc1F)Nc1ccc(Oc2ncccn2)cc1', 'C=CCn1c(=O)c(C)nnc1SCCC(=O)Nc1ccc(C#N)cc1', 'Clc1ccccc1Oc1cc2c(cc1OCCCOCCO)C(=O)CC2', 'ClCc1cc(CCO)ccc1F', 'Clc1cc(N2CCCCC2)c([N+](=O)[O-])cc1NCC1CCCC1', 'O=C(c1ccsc1)Nc1ccn(CCOc2ccccc2F)n1', 'OC1(SCCC(=O)Nc2ccc(Br)cc2Cl)CCCCC1', 'Clc1ccccc1C(C)(C)NCc1nc(C2CC2)n2ccccc12', 'Clc1ccccc1[C@H](NS(=O)(=O)c1ccc(Cl)cc1)c1cccc2ccccc12', 'O=C(Cc1nc([O-])c(-c2ccccc2Cl)n1C1CC1)Nc1ccccc1', 'Clc1ccc(Cc2nnn(-c3ccccc3)n2)cc1Cl', 'Clc1ccccc1-c1nn(Cc2ccnc(COC(F)(F)F)c2)nc1-c1ccc(Br)cc1', 'Oc1cc(-n2nnc(/N=C/c3ccccc3)c2-n2cncc2C(F)(F)F)ccc1F', 'Clc1cc(/C=N/N2CCOCC2)ccc1OC(=O)Cc1ccccc1', 'Oc1cc(O)ccc1OCC(=O)NCC(=O)c1ccccc1O', 'NS(=O)(=O)CCc1nc(-c2ccccc2)cs1', 'Oc1ccc(F)cc1-n1c(=S)[nH]c2cc(Cl)ccc21', 'C[C@@H](Oc1ccc(Cl)cc1)C(=O)N1CCN(S(=O)(=O)c2ccccc2)CC1', 'Clc1cc(Br)c(CN2CCN(c3cnc4cc(Cl)ccc4c3)CC2)cc1', 'O=C(Cc1coc2ccccc12)NCc1cccc(-n2cccn2)c1', 'O=[N+]([O-])c1ccncc1CN1C(=O)c2ccccc2C1=O', 'Clc1ccc(Br)cc1-c1cnn(CCc2ccccc2)n1', 'Clc1ccc(-c2csc3sc(CCNC(=O)Cc4ccccc4)nc23)cc1', 'O=C(Cc1coc2ncccc12)NCCn1nc(-c2ccccc2)ccc1=O', 'Clc1ccc(Br)cc1-c1cnn(CCN2CCCC2)c1', 'CCNc1ccc(C(=O)N[C@@H](C)c2ccccc2Br)cc1OC', 'OC[C@H](CO)Nc1ccc(F)c(Cl)c1', 'Clc1ccccc1C1(Oc2ccccc2F)CC1', 'ClCc1csc(N2CCO[C@@H](c3ccccc3)C2)c1Cl', 'Clc1ccc(-c2cccs2)nc1Cl', 'C=CCOCCCNC(=O)CNC(=O)c1cccc(C(F)(F)F)c1', 'Oc1cc(OCCSc2ncc(-c3ccccc3)n2Cc2ccccc2)ccc1Oc1ccccc1', 'CN(c1ccn(-c2ccccc2Cl)n1)C1CCCC1', 'Clc1ccc(CO)c(Oc2ccccc2Br)n1', 'O=C(CN(C(=O)c1cccc(Cl)c1)N1CCCC1)c1ccccc1', 'Clc1cc(Br)cnc1Oc1ccccc1[N+](=O)[O-]', 'Clc1ccccc1CNc1cnn(-c2cc3ccccc3nc2Cl)c1', 'Clc1ccccc1O/Cn1nccc1COc1ccc(Cl)cc1Cl', 'Clc1ccc(-c2nnnn2C[C@H](O)c2ccccc2)cc1', 'Clc1cccc(Nc2nccs2)c1Cl', 'ClCc1csc(-c2cc(CN(Cc3ccccc3)c3ccccc3)oc2C2CC2)c1Cl', 'O=C(c1cc(F)c2ccccc2c1Cl)Nc1ccc2c(c1)OCO2', 'Oc1ccc([N+](=O)C2C[C@@H](C3CCN(c4ccccc4F)[C@@H]3c3ccccc3)c3ccccc32)cc1', 'Clc1ccccc1CO[C@H](CO)c1ccc(Cl)cc1', 'O=C(/N=C\\c1ccc(S(=O)(=O)N2CCCC2)cc1)c1ccccc1Cl', 'Oc1cccc(NCc2ncc(-c3ccccc3)o2)c1O', 'O=C(Cc1c(F)cccc1F)Oc1ccc2c(c1)OCCO2', 'O=[N+]([O-])c1cc(Cl)ccc1Oc1ncccn1', 'Oc1ccc(/C=C(\\c2ccccc2)C(=O)Nc2cc(C(F)(F)F)c(Cl)c(Cl)c2)cc1', 'Oc1ccccc1-n1nnnc1SCCCCC(=O)NCCc1ccccc1', 'Clc1ccccc1O/N=C/c1ccccc1O', 'Clc1ccccc1-c1csc(/C=C/c2ccncc2)n1', 'Oc1ccc(F)c(Cl)c1CCc1ccc2ccccc2n1', 'Clc1cccc(/N=N/CCN2CCN(c3cccc(F)c3)CC2)c1', 'O=C(c1cn[nH]c1C(=O)c1ccccc1)N1CCOCC1', 'Clc1ccccc1Oc1cc(F)c(Cn2ccc3[nH]ccc32)cc1', 'Clc1ccccc1OCCCNCc1csnn1', 'Clc1cccc2ncn(Cc3cc(F)cc(F)c3)c12', 'Clc1ccc(CN[C@@H](c2nc3ccccc3s2)c2ccccc2)cc1', 'Clc1ccccc1C(=O)O[C@@H]1CN(C2CC2)CCN1Cc1ccccc1', 'Clc1ccc(-c2cc(Br)nc(N3CCC(CN4CCCC4)CC3)n2)cc1', 'Cc1ncccc1C(=O)Nc1cncc(C(F)(F)F)c1', 'ClC[C@H]1CCCN(c2ccccc2)C1', 'Nc1[nH]ncc1Nc1ccc(C(=O)Nc2ccccc2F)cc1Cl', 'Oc1ccccc1OC(=O)CN1CCc2cc(F)ccc2C1', 'Clc1ccc(Br)c(Cl)c1C(=O)Nc1ccc(C(F)(F)F)cc1', 'ClCc1csc(OCc2ccccc2)n1', 'ClCc1csc(OCc2ccccc2)n1', 'O=C(C[NH+]1CC[C@@H](c2nc3ccccc3s2)C1)NCCOc1ccc(N2C(=O)NC(=O)c3ccccc32)cc1', 'O=C(c1cccc2ccccc12)N1CCO[C@@H]2CCCC[C@@H]21', 'Clc1cc(COC(=O)c2ccco2)ccc1[N+](=O)[O-]', 'Clc1ccc(-c2n[nH]c(NC3CCCC3)n2)cc1', 'Clc1ccccc1CSc1nccc(C(F)(F)F)n1', 'O=C(C1CCN(c2ncccc2Cl)CC1)N1CC=C(c2ccccc2)CC1', 'Clc1cccc(-c2ccoc2Br)c1', 'O=[N+]([O-])c1ccncc1Cc1cccc(F)c1', 'Oc1cc(-n2cnc3ccccc32)cc([N+](=O)[O-])c1', 'O=C(CCc1cc2ccccc2oc1=O)OCc1cccnc1', 'Clc1ccc(COc2ncccc2F)cc1Br', 'Nc1c(CSc2ccccc2Cl)cnn1-c1ccc(Cl)cc1', 'Clc1ccccc1Oc1cnc(F)c(Cl)c1', 'Clc1cc(/C=C(\\C#N)C(=O)Nc2ccc(I)cc2)ccc1Cl', 'Oc1ccccc1/N=C(\\NCC(=O)N1CCN(c2ccc(Cl)cc2)CC1)C(=O)[O-]', 'Oc1ccc(Oc2ncnc3sccc23)c2ccccc12', 'Clc1ccccc1Oc1cc(S(=O)(=O)N2CCN(CC(F)(F)F)CC2)ccc1Cl', 'CNS(=O)(=O)c1ccc(=O)n(CCC(=O)N(C)Cc2ccc(Cl)cc2)c1', 'O=[N+]([O-])c1ccncc1-c1ccc([N+](=O)[O-])cc1', 'O=C(CSC(c1ccccc1)c1ccc(Cl)cc1)Nc1ccccc1C(=O)N1CCCC1', 'Clc1cc(Cl)nc(COc2ccccc2F)n1', 'CNS(=O)(=O)c1ccccc1NC(=O)CCN1Cc2ccccc2OC1', 'O=C(CSCCOCc1ccccc1)Nc1n[nH]c2ccccc12', 'Clc1cccc(Br)c1Oc1cccc2c1CCC2', 'O=[N+]([O-])c1cc(N2CCOCC2)ccc1N1CCOCC1', 'Clc1cc(Cl)c(CN2CCOCC2)cc1O', 'O=C(c1ccco1)N1CCOc2cc(-c3ccccc3F)ccc21', 'O=C(c1cc(Cl)ccc1[N+](=O)[O-])N1CCN(S(=O)(=O)c2cccc([N+](=O)[O-])c2)CC1', 'Clc1ccc(-c2csc3sccc23)cc1', 'O=C(/C=C/OCc1ccccc1O)c1ccccc1F', 'Clc1cc(/C=C2\\CCc3ccccc32)ccc1O', 'Clc1ccc(Cc2nc(C(F)(F)F)cs2)cc1', 'Clc1ccccc1Nc1cc(C2CC2)ncn1', 'Clc1ccc(CO)cc1OCCn1nccc1CN1CCN(Cc2ccccc2)CC1', 'Clc1cc(N2CCc3ccccc32)cc([N+](=O)[O-])c1', 'Oc1cc(OCCOCOc2ccccc2)ccn1', 'O=C(Cc1coc(-c2ccccc2Cl)n1)NC1COC1', 'Clc1ccccc1-c1noc(C(=O)N2CCC(Cc3ccco3)CC2)n1', 'Clc1cccc(S(=O)(=O)Nc2ccc3c(c2)OCCO3)c1O', 'Clc1ccccc1OCc1csc(OC2CC2)n1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1-c1ccc(F)cc1', 'Oc1ccc([N+](=O)[O-])c(C(=O)c2ccc(NCCO)cc2)c1', 'O=[N+]([O-])c1c(F)c(Cl)cnc1NCc1ccc(N2CCOCC2)cc1', 'Clc1cc(N2CCc3c(c(OCc4ccccc4)nn3Cc3cccc(F)c3)C2)ccc1[N+](=O)[O-]', 'O=C(COc1cc(C2CCN(c3ncccn3)CC2)on1)NCc1ccccc1', 'Clc1ccc(-c2cc(Cn3cnnn3)ccn2)cc1', 'O=C(/C(Cc1ccccc1)Cn1cccn1)c1ccc(Cl)cc1', 'C[C@@H](N[C@@H](CO)C(C)C)c1ccccc1', 'Clc1nnc(CSc2ccc(N3CCCC3)cc2)nc1N1CCCC1', 'Clc1cc(Br)c2nc(-c3ccccc3)[nH]c(=O)c2c1', 'Oc1ccc(Oc2cn(Cc3ccccc3)nn2)cc1Cl', 'Clc1ccc([C@@H](NC[C@@H]2CCCCO2)c2ccc(Cl)cc2)cc1', 'ClC[C@H]1CN(c2ccccc2)CC1', 'O=C(/C=C\\Cc1ccc(Oc2ccccc2)c(F)c1)c1cc(O)ccc1F', 'CN[C@H](c1ccccc1)c1ccc(F)cc1Cl', 'Clc1ccc(-c2csc3[nH]c(Nc4ccccc4)cc23)cc1', 'O=[N+]([O-])c1ccc(-c2noc(-c3ccc([N+](=O)[O-])cc3)n2)cc1Cl', 'Oc1cc(Br)cc2cc(F)ccc12', 'Oc1cc(-n2cc(-c3ccccc3)nn2)ccc1Br', 'O=[N+]([O-])c1c(-c2cccnc2)ncc(C(=O)Nc2ccccc2F)c1C1CC1', 'Clc1ccc(-c2cccnc2N=C2CCCCC2)cc1', 'Clc1ccccc1COCCOC[C@@H]1CCCO1', 'Clc1cc(Cl)nnc1-c1ccccc1Cl', 'Clc1ccc(Cc2ccc(OCc3ccc(Cl)cc3Cl)o2)cc1', 'Oc1cc(-n2nnc(-c3ccccc3)n2)c2ccccc2c1OCC[C@@H]1CCCC[NH+]1CC1CC1', 'O=C(Cc1noc2ccccc12)Nc1cccc(Sc2nc3ccccc3s2)c1', 'O=C(CNc1c(F)cc(F)cc1Cl)Nc1ccc(F)c(F)c1', 'Clc1ccc(Br)c(Cl)c1Cl', 'Clc1ccc(Br)c(Cl)c1Cl', 'Clc1ccc(Br)c(Cl)c1Cl', 'Clc1ccc(-c2n[nH]c([C@H](c3c[nH]c4ccccc34)N3CCO[C@@H](C(F)(F)F)C3)c2)cc1', 'Clc1cccc(-c2cncc(Br)n2)c1', 'N=Cc1cnn(-c2ccc(F)cc2F)c1NC(=O)C(=O)NC(=O)c1ccc2ccccc2c1', 'Clc1ccc(Cc2nc3ccccc3[nH]2)nn1', 'O=[N+]([O-])c1ccc(OCC(=O)N2CCCC[C@H]2c2ccccc2)cc1', 'Clc1ccc(-c2cccs2)nc1CN1CCCc2ccccc21', 'O=[N+]([O-])c1ccncc1[C@@H]1CN(Cc2ccncc2)C1', 'Oc1cccc2cc(OCC(=O)Nc3cc(C(F)(F)F)ccc3O)ccc12', 'OC[C@@]1(Oc2ccccc2-c2ccccc2)COc2ccccc21', 'NS(=O)(=O)N(C)C1CCC(Oc2ccc(N(C)C)cc2)CC1', 'O=C(Cc1n[nH]c2ccccc12)Nc1ccc2c(c1)CCO2', 'O=C(c1c[nH]c2ccccc12)N1CCC(OC[C@H]2CCCCO2)CC1', 'Clc1ccccc1Nc1cnc(F)cc1Cl', 'O=C([O-])/C=C/C(=O)N(Cc1ccccc1)c1ccccc1', 'COc1nnnc(Nc2ccc(OC)cc2)c1C', 'O=C(c1cn[nH]n1)N1CCC(Oc2ccccc2)CC1', 'O=[N+]([O-])c1ccncc1Nc1ccc([N+](=O)[O-])cc1', 'Clc1cc(Br)ccc1OCC(=O)N[C@H]1c2ccccc2C1', 'O=C([O-])Cc1ccc(NS(=O)(=O)c2ccccc2F)cc1', 'Clc1cccc(Br)c1Oc1ccc2c(c1)OCO2', 'Clc1ccccc1Oc1cc(C(F)(F)F)nc(Nc2ccc(F)cc2)n1', 'Oc1cc(OCOCc2nc3ccccc3s2)ccc1N1CCOCC1', 'Clc1cc(/C=N/N2CCN(C3CCCCC3)CC2)ccc1Br', 'Clc1nnnn1CCc1ccccc1', 'Clc1ccc(-c2nnnn2/N=C\\c2ccccc2Cl)cc1', 'Clc1ccc(-c2nnnn2/N=C/c2ccccc2Cl)cc1', 'Clc1ccccc1Oc1cnc(NN2CCOCC2)nc1', 'O=C(/C=C1\\C(=O)Nc2ccccc21)c1ccccc1Cn1cccn1', 'Clc1ccccc1Nc1cccc(C(=O)Oc2ccc(C(F)(F)F)cn2)c1', 'O=C(CS(=O)(=O)c1ccccc1Cl)Nc1cc(Cl)cc(Br)c1', 'Clc1cc(Cl)cnc1C1CCCCC1', 'Clc1ccccc1-c1csc(C(F)(F)F)n1', 'Clc1ccc(-c2csc3scc(-c4ccc(Cl)c(F)c4)c3c2=O)cc1', 'ClCc1cncc(-c2ccc(O)c3ccccc23)n1', 'CCn1nnc(CNC(=O)c2ccc(F)cc2Cl)n1', 'Oc1cc(F)cc(F)c1C(=O)Nc1ccc(C(=O)NC2CC2)cc1[N+](=O)[O-]', 'Clc1ccc(Br)cc1OS(=O)(=O)Cc1ccccc1F', 'ClCc1cc(F)ccc1C1CC1', 'O=C(CNCc1nnnn1-c1cc(F)cc(Cl)c1)NCc1ccc(F)cc1', 'Oc1ccc([N+](=O)[O-])c(-c2ccc(F)cc2F)c1', 'O=[N+]([O-])c1cc(F)c2cccnc2c1Oc1ccccc1', 'Clc1ccc(CO)cc1OCc1cscn1', 'CCCn1nnnc1COc1cccc(Br)c1C', 'O=C([C@H](CSc1ccccc1)c1ccccc1)N1CCCC1', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccccc1O', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccc(F)cc1F', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccc(F)cc1F', 'COc1ncnc(Nc2cc(C)cc(C)c2)c1C', 'Clc1nncc(-c2ccccc2Cl)n1', 'Clc1cc(/C=C/N=C(\\Nc2ccccc2)c2ccccc2)ccc1Cl', 'Oc1ccccc1-n1nnnc1SCc1ccc(Cl)c(Cl)c1', 'N[C@@H](NC1CCN(c2ccccc2)CC1)c1ccccc1', 'Clc1nncc(COc2ccccc2CN2CCOCC2)n1', 'Clc1ccc([C@@H](O)c2ccccc2)nc1', 'Clc1ccsc1S(=O)(=O)Nc1ccccc1C(=O)c1ccc(Br)cc1', 'Clc1ccc(Br)cc1-n1cncn1', 'Clc1cccc(F)c1[C@@H]1CCCN1Cc1ccccc1F', 'O=[N+]([O-])c1c(F)ccc2c1N(Cc1ccc(Br)cc1)CC2', 'Clc1ccccc1CSC[C@H](O)COc1ccc(F)cc1', 'Clc1ccc(-c2cc(S(=O)(=O)N3CCN(c4ccccc4)CC3)co2)cc1', 'Clc1ccc(-c2cc[nH]n2)c(NCc2ccccc2)c1', 'O=C(Cc1nc(Cl)ccc1Cl)c1cccc(Cl)c1', 'Clc1cccc(Br)c1Sc1cccc(Br)c1', 'O=C(Cc1c(-c2ccccc2)nn([O-])c1Cl)C(=O)N1CCN(c2ccccc2)CC1', 'Clc1cc(Cl)ncc1N1CCc2ccccc21', 'O=C(c1c[nH]c(Nc2cc(Cl)ccc2F)c1)c1ccccc1', 'Clc1ccccc1O/Cn1cnc(-c2ccc(Cl)c(F)c2)n1', 'Clc1ccc(Br)cc1OS(=O)(=O)c1ccccc1[N+](=O)[O-]', 'Clc1ccc(Cc2ncc[nH]2)cc1F', 'Clc1cccc(Br)c1Cn1nnc2cc(F)ccc21', 'Clc1ccccc1OCc1ccn(CCCOC(F)F)n1', 'O=C(c1cncc(Cl)c1)C1CCN(c2cc(-c3ccccc3)n[nH+]c2-c2cccs2)CC1', 'O=C(Cc1n[nH]nc1C(F)(F)F)Nc1cnn(Cc2ccccc2)c1', 'Oc1ccc([N+](=O)c2ccccc2)c(F)c1Oc1ccccc1F', 'COc1nc2nc(C(=O)NCCCc3ccc(F)cc3)cn2cc1[N+](=O)[O-]', 'CNS(=O)(=O)CCCNC(=O)/C=C/c1cccc2ccccc12', 'Clc1cc(/C=Cc2ccccn2)ccn1', 'O=C(COC(=O)c1cnc(Br)cn1)Nc1cc(Br)ccc1Cl', 'Clc1ccc(Br)cc1OS(=O)(=O)c1cc(F)ccc1F', 'Clc1ccc(-c2nnnn2-c2cc([N+](=O)[O-])ccc2F)cc1', 'Clc1ccc(-c2nn(-c3ccc([N+](=O)[O-])cc3)c(Cl)c2Cl)cc1', 'Oc1cccc(-c2csc(CN(c3ccc(Cl)cc3)S(=O)(=O)c3ccccc3)n2)c1', 'O=[N+]([O-])c1cc(Nc2cc([N+](=O)[O-])c(Cl)cc2)ccc1Br', 'OCc1n[nH]c(Nc2ccc(F)cc2)c1-c1ccccc1', 'Clc1cccc(/N=C/c2cccc(-n3cccn3)c2)c1', 'ClCc1cnc(Cl)n1Cc1ccccc1', 'Clc1nnc(NCc2cccc(F)c2)n1-c1ccccc1', 'Clc1cc(/C=C2\\C(=O)NC(=S)N(c3ccccc3)C2=O)ccc1Cl', 'NS(=O)(=O)CCNc1ccc(Oc2cccc(C#N)c2)cc1', 'Clc1ccsc1S(=O)(=O)N1CCN(c2ccccc2)CC1', 'Clc1cccc2cc(CNCc3ccccc3F)[nH]c(=O)c12', 'Clc1ccsc1S(=O)(=O)Nc1ccc(C(=O)N2CCOCC2)cc1', 'Clc1ccccc1Oc1cnc(N2CCOCC2)c2cc(Br)ccc12', 'O=C(COc1ccccc1Cl)N[C@H](Cn1cc[nH+]c1)c1ccccc1', 'Oc1cc(OCOCCOc2ccc(C(F)(F)F)cc2)ccc1Br', 'Clc1nnnc(-c2ccccc2)n1', 'O=C(Cc1cc(=O)n2ccsc2n1)Nc1ccccc1N1CCCC1', 'NS(=O)(=O)Nc1cc(C(=O)N2CCc3ccccc3C2)ccc1O', 'Clc1ccc(-c2nonc2N2CCOCC2)cc1', 'COC[C@@H](O)CN1CCN(C(=O)NCCc2ccc(F)cc2)CC1', 'Clc1ccccc1Oc1cnc(C(C)C)cc1O', 'Clc1ccccc1N1N(CCOc2ccc(Br)cc2Cl)CCCC1', 'O=C(CSCN1C(=O)CCc2ccccc21)NCCCc1ccccc1', 'Oc1ccccc1CCn1nc(-c2cnccn2)ccc1=O', 'O=C(CSCc1cccc(Br)c1Cl)Nc1ccccc1C(=O)NCc1ccccc1', 'O=C(CC1(O)CCOCC1)N1CCN(Cc2cccc(C(F)(F)F)c2)CC1', 'Clc1cc(Cl)c(C(=O)Br)cc1Oc1ccc(F)cc1', 'OCc1nc(-c2ccccc2Br)cs1', 'Clc1cc(CCOc2ccccc2)sc1NCc1ccccc1F', 'O=C(c1cncc(I)c1)N1CCC(OCc2ccccc2F)CC1', 'Clc1ccc(Br)c(F)c1Oc1cccc(CO)c1', 'Clc1ccccc1OCCOCCN1C(=O)NC2(CCCCC2)C1=O', 'Oc1ccccc1-n1nnnc1SCCCCCO', 'O=[N+]([O-])c1ccc2[nH]ncc2c1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1', 'ClCc1cc(Br)ccc1O', 'Clc1cc(Nc2c([N+](=O)[O-])cc(Cl)cn2)ccc1F', 'Clc1ccc(C2CC2)cc1NCc1nc(C2CC2)no1', 'Oc1ccc(F)cc1OCc1ccn(-c2cccc(F)c2)n1', 'Clc1ccccc1N1N(C2(NS(=O)(=O)c3ccccc3)COC2)SC1=O', 'Clc1nnnn1c1c(NC2CCN(c3ccccc3)CC2)ncn1-c1ccccc1', 'O=[N+]([O-])c1c(Br)scc1C(=O)NCCNC(=O)c1ccccc1F', 'CNS(=O)(=O)c1ccc([N+](=O)[O-])cc1N', 'Oc1cccc(Br)c1Cl', 'O=C(CCc1ccsc1)Nc1ccc2nc(C3CC3)oc2c1', 'Clc1cccc(/C=C/CCn2cccc(C(F)(F)F)c2=O)c1', 'OCc1ncn(-c2ccccc2)n1', 'Oc1cccc(-c2cc(F)c(Br)c(Cl)c2)c1', 'COc1nc(Nc2nc(C3CC3)n(-c3ccccc3C)n2)cc2c1OCO2', 'Oc1ccc(F)cc1OS(=O)(=O)c1ccccc1O', 'Clc1cc(Cl)c2c(c1)OCO2', 'O=C(CNS(=O)(=O)C(F)(F)F)Nc1ccccc1Br', 'Oc1ccc(/C=N/Nc2nccc(N3CCCC3)n2)cc1Cl', 'Oc1ccc(/C=N/S(=O)(=O)c2ccc([N+](=O)[O-])cc2)cc1', 'O=C(COc1cc(F)c(Br)cc1F)N1CCC[C@@H]1c1ccccc1', 'Oc1cc(Br)cc(N2CCN(CC3CC3)CC2)c1', 'ClC[C@@H]1CCN(Cc2ccc(F)cc2)C1', 'O=[N+]([O-])c1cc(Nc2ccc(-n3cccn3)cc2)cc(S(=O)(=O)N2CCCC2)c1', 'Clc1ccc(Cc2noc(C[C@@H]3CCCN3C(=O)c3ccccc3)n2)cc1', 'Clc1ccc(Br)cc1-c1cc(-c2cccnc2)ncn1', 'COCc1nnc(NC(=O)CCc2c(C)cccc2C)s1', 'Clc1ccc(COc2nc(Cl)cc(Cl)c2Cl)cc1', 'NS(=O)(=O)CCCc1cccc(C(=O)N(C2CC2)C2CCCCC2)c1', 'O=C(CCCSC1CCCC1)NCc1nc2ccccc2[nH]1', 'C[C@@H](c1ccc[nH]1)N(C)C(=O)[C@@H]1CC(=O)N(Cc2ccccc2F)C1', 'Oc1cc(Cn2nc3ccccc3n2)ccc1Cl', 'Oc1ccc(/C=N/Oc2cccc(Br)c2)cc1', 'Oc1ccccc1-n1nnnc1SCc1ccc(Cl)cc1Cl', 'COc1nn(C)cc1C(=O)Nc1nc(-c2ccccc2)c(C)s1', 'Clc1ccc(-c2nonc2COC2CC2)cc1', 'Clc1ccccc1Oc1cc(Br)c([N+](=O)[O-])cc1Br', 'Oc1cc(F)c(C(=O)NCC2(c3ccccc3)CCOCC2)cc1Cl', 'Clc1ccccc1O/Cn1[nH+]c(-c2ccccc2)nc1-c1ccccc1Cl', 'Clc1cccc(NCCc2nccc(-c3cccc([N+](=O)[O-])c3)n2)c1', 'Oc1cc(Cn2ncccc2=O)ccc1COC1CCN(C(=O)c2ccccc2)CC1', 'Clc1cccc2c1N(Cc1cnn(Cc3ccccc3F)c1)CCC2', 'Clc1cc(Br)c(CNc2ccc3c(c2)CCCO3)cc1Cl', 'Nc1[nH]nc(Sc2ccccc2)n1', 'CCc1ccc(-c2csc(=S)n2C2CCCCC2)cc1', 'Clc1nnnc(NC2CCC(Oc3ccccc3)CC2)c1C1CC1', 'Oc1cccc2cc(C(=O)OCCN3CCOCC3)cnc12', 'Clc1ccccc1OC1(NC(=O)C(=O)c2ccccc2)CC1', 'Cc1ncsc1C(=O)Nc1ccccn1', 'Clc1ccccc1NS(=O)(=O)c1c[nH]c(=O)[nH]c1=O', 'O=[N+]([O-])c1c([N+](=O)[O-])ccn1Cc1ccccc1', 'O=C(c1ccsc1Cn1cnnn1)NC(c1ccccc1)c1ccccc1', 'ClCc1csc2ncn(Cc3ccc(Oc4ccccc4)cc3)c12', 'O=[N+]([O-])c1ccn(Cc2cccc(C(=O)N3CCCC3)c2)c1', 'Clc1cc(Cl)ncc1Cl', 'O=C(c1cc(Cl)cnc1N1CCCCC1)N1CCN(C(=O)c2cccnc2)CC1', 'COc1ncncc1CNC(=O)C(=O)Nc1cccc(OC)c1', 'O=[N+]([O-])c1ccnc2cccnc12', 'O=[N+]([O-])c1ccnc2cccnc12', 'O=[N+]([O-])c1ccnc2cccnc12', 'O=[N+]([O-])c1ccnc2cccnc12', 'Clc1cccc(F)c1Nc1ccc([N+](=O)[O-])cc1[N+](=O)[O-]', 'N=CCc1ccc(Cl)c(CCC(=O)Nc2ccc(F)cc2)c1', 'OCc1nc2ccccc2c(=O)n1-c1sc2c(c1-c1ccccc1)CCCC2', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1Nc1cccc(Cl)c1', 'Oc1ccc(/N=c2/[nH]ccn2CCOc2ccccc2)cc1', 'Oc1cc(CCn2nc(-c3cc(F)ccc3F)nn2)ccc1Cl', 'Clc1ccc(Br)cc1S[C@H]1CCOCC1', 'O=C(Cc1cccc(Cl)c1)NCc1cccc(-c2ncnn2C2CC2)c1', 'Clc1cccc([S@@]c2ncc(Br)cc2Cl)c1', 'OC[C@@]1(NC2CC2)Cc2ccccc2C1', 'CNS(=O)(=O)CCc1ccc(NC(=O)N2CCc3sccc3C2)cc1', 'Oc1cccc2cc(C(=O)N3CCCCC3)c3nc(-c4ccccc4)c(=O)[nH]c3c12', 'O=C(/C=C/c1cccnc1)Nc1ccc([N+](=O)[O-])c(Br)c1', 'Oc1ccc(O[C@@H](CO)C(=O)NCc2ccccc2F)cc1', 'O=C(/C=C/c1ccc2c(c1)OCO2)NCc1cccs1', 'Cc1nnsc1CN1CCN(C(=O)c2ccc(Cl)cc2)CC1', 'Clc1ccc(Br)cc1-c1cc2ncccc2nc1Cl', 'Clc1cc(Cl)ncc1/C=C/C(=O)C(c1ccccc1)c1ccccc1', 'Clc1ccc(-c2cccnc2N2CCSCC2)cc1', 'Clc1nnnn1-c1c(-c2ccc(F)cc2)[nH]c2ccccc21', 'ClCc1ccnc(-c2noc(-c3ccccc3)n2)n1', 'Oc1cc(-n2nncc2CN2CCN(Cc3ccccc3)CC2)ccc1[N+](=O)[O-]', 'Clc1ccc(Br)cc1-n1cnc(Cc2ccccc2)n1', 'Clc1cc(Cl)nnc1Nc1ccc(Br)cc1', 'O=C(COCCN1CCCC1=O)Nc1cc(Br)cc(Cl)c1', 'Oc1cc(O[C@@H](CO)c2ccccc2)ccc1-c1ccc(Cl)cc1', 'C[C@@H](NS(=O)(=O)c1ccccc1F)C(=O)NCc1ccc(F)cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(Cl)c1Oc1ccccc1Br', 'N#Cc1c(Cl)cc(Cl)cc1C(=O)NCc1cccc(Br)c1', 'O=C(Cc1cnco1)Nc1cccc2c1CCCC2', 'Oc1cccc(Br)c1Cn1cccc(Br)c1=O', 'O=C(CNS(=O)(=O)c1ccc(Cl)s1)Nc1cc(Cl)c(Cl)cc1Cl', 'COc1ncncc1CNCc1cc(F)ccc1F', 'O=C(c1cc(Br)c[nH]1)N(Cc1ccncc1)Cc1cccnc1', 'N#CCc1ccc(CNC(=O)Nc2nc(C3CC3)n(-c3ccccc3)n2)cc1', 'O=[N+]([O-])c1ccncc1Oc1ccc(N2CCOCC2)nc1', 'O=C(CSCc1ccc(Cl)c(Cl)c1)NNC(=O)c1ccc(S(=O)(=O)N2CCCC2)cc1', 'Clc1ccc(Br)cc1-c1cnn(-c2ccc(F)cc2)n1', 'Clc1ccc(Br)cc1-c1cnn(-c2ccc(F)cc2)n1', 'Clc1ccsc1/CCNC(=O)NCCCN1CCCCC1=O', 'Clc1ccccc1Cc1c(Cl)nc(Cl)nc1Cl', 'Oc1cc(-n2nnnc2SCc2ccccc2)ccc1OCc1cccnc1', 'O=C(CNCc1nccs1)N1CCC(Oc2ccc(F)cc2F)CC1', 'O=C(CSCC(=O)N1CCCCC1)NNC(=O)c1ccccc1C(F)(F)F', 'Clc1ccccc1CCNCc1cc(=O)n2ncccc2n1', 'Oc1cccc(Br)c1Oc1ccccc1C(F)(F)F', 'Clc1ccc(C(N)=O)c([N+](=O)[O-])c1Sc1ccccc1', 'Clc1ccc(Br)c(Sc2nnnn2-c2ccccc2)c1', 'O=C(C1CCC1)N1CCN(C(=O)c2cnc(CN3CCOCC3)s2)CC1', 'Cc1ncsc1CN(C)c1ccc(C(=O)N2CCCC2)cc1', 'NS(=O)(=O)Nc1ccc([N+](=O)[O-])cc1Cl', 'Clc1ccccc1OCCn1ncnc1-c1cc(Cl)ccc1O', 'ClCc1nonc1COc1ccccc1', 'Clc1ccc(C2(OC[C@@H]3CCN(C4CCCCC4)C3)CC2)cc1', 'Clc1ccc(Br)cc1-n1nnc2ccc([N+](=O)[O-])cc21', 'O=C(CNc1cccc(C(F)(F)F)c1)NC(=O)c1ccc(Cl)nc1', 'Clc1cc(/C=N\\O)ccc1F', 'OC[C@@H](c1ccccc1)N1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1', 'COC(=O)[C@@H](C)NS(=O)(=O)c1ccccc1Cl', 'Clc1cc(F)ccc1[C@H](O)c1ccccc1', 'O=[N+]([O-])c1cc(CNc2ccc(Oc3ccccc3)nn2)cc2c1OCCO2', 'COc1nn(-c2cccc(C(=O)Nc3cc(Cl)c(OC)cc3)c2)c(C)c1Br', 'Oc1cccc(CC[C@@H]2CCCCO2)c1', 'O=C(/C=C\\c1cccnc1)NNc1cccc(NC(=O)c2cccnc2)c1', 'Clc1cc(Cl)c(COC2CCCCC2)cc1O', 'O=C(CCc1n[nH]c(=O)c2ccccc12)NNC(=O)c1ccc(F)cc1F', 'Clc1ccccc1-c1nnc(CSc2ncccn2)o1', 'Clc1ccc([C@@H](O)c2ccc(Br)cc2)c(F)c1', 'NS(=O)(=O)/C=C/c1ccc(F)cc1', 'ClCc1nonc1COC(c1ccccc1)c1ccccc1', 'Clc1ccccc1CO[C@H]1CCCOC1', 'Clc1ccc(Br)c(OCc2nnc3ccccn23)c1', 'Clc1ccccc1NC(=S)Nc1nncs1', 'Clc1cccc2nc(N3CCCC3)[nH]c12', 'Clc1ccc(Br)c(-c2ccco2)c1N', 'Clc1cccc(F)c1CCNC[C@@H]1CCc2ccccc21', 'Clc1cccc([S@@]c2nc3cc(Cl)ccc3[nH]c2=O)c1', 'O=C(COc1ccc2c(c1)OCOC2)Nc1cccc(F)c1Cl', 'O=C(c1cccs1)c1ccnc(OC2CCCCC2)c1', 'ClCc1ccn(-c2cccc(Br)c2)n1', 'Oc1ccccc1-n1nnnc1COc1ccc2nccnc2c1', 'Clc1nnn(-c2ccccc2)c1Cn1ccnc1-c1cnccn1', 'Clc1ccsc1-c1cccc(C(F)(F)F)c1', 'O=C(COc1cc(N)c(C(C)C)cc1[N+](=O)[O-])N1CCCC1', 'Clc1ccc(Br)cc1OS(=O)(=O)c1c(Cl)cccc1Cl', 'Oc1ccccc1/N=C(\\Nc1c(F)cc(Cl)cc1)c1ccc(Cl)cc1', 'Clc1ccccc1N1N(CC(=O)Nc2cccc(Cl)c2)CC1', 'CCCCSC[C@@H](C)CNC(=O)c1cccc(Br)c1', 'OC1=C(c2ccccc2)/C(=C/c2ccc(F)cc2)C1', 'Clc1ccc(-c2nn(-c3cc([N+](=O)[O-])ccc3Cl)nn2)cc1', 'Clc1ccc(Br)c(S(=O)(=O)N2CCN(c3ccc(Cl)c(C(=O)[O-])c3)CC2)c1Cl', 'O=[N+]([O-])c1cc(Cl)nc(Nc2ccccc2Cl)c1', 'Clc1cccc(NCC2(O)CCOCC2)c1', 'Clc1ccccc1Nc1ccc2nnc(-c3cccs3)n2n1', 'O=[N+]([O-])c1cc(N2CCN(Cc3ccccc3N3CCOCC3)CC2)nc(-c2ccco2)n1', 'O=C(/C=C/OC1CCCC1)Nc1ccccc1[N+](=O)[O-]', 'O[C@@H]([C@@H](N[C@@H](CO)c1ccccc1)c1ccccc1)c1ccccc1', 'Nc1[nH]ncc1-c1ccc([N+](=O)[O-])cc1', 'Clc1cc(/C=C/N=C(\\c2ccccn2)c2ccccc2)cc(Cl)c1', 'Clc1cccc(NCc2ccc[n+]([O-])c2)c1', 'O=[N+]([O-])c1ccnc2cccc(-c3nc4ccccc4s3)c12', 'ClCc1ccc[nH]c1=O', 'ClCc1ccc[nH]c1=O', 'Clc1ccccc1C1CCN(S(=O)(=O)c2cc(OC(F)(F)F)ccc2Cl)CC1', 'Clc1cccc(CCc2nnc(Cc3ccsc3)n2Cc2ccccc2)c1', 'Clc1ccsc1N/C(=C/c1ccc(Cl)cc1)NCc1ccccc1', 'Clc1cccc2cc(/C=C/Br)ccc12', 'Clc1ccccc1CNC1=N/C(=C/c2ccncc2)N1c1ccccc1', 'Clc1ccc(-c2nonc2NCCN2CCOCC2)cn1', 'ClCc1cccc(Cl)n1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1-c1csc(-c2ccccc2)n1', 'Clc1ccc(-c2cc[nH]n2)cc1', 'Clc1ccc(-c2cc[nH]n2)cc1', 'Clc1ccc(-c2cc[nH]n2)cc1', 'Clc1cccc(F)c1OCC(=O)NCc1ccc(C(=O)[O-])cc1', 'O=C(C1CCCCC1)N1CCC(N2CCOCC2)CC1', 'Oc1cc(CCNCc2nc3cc(Cl)ccc3s2)ccc1F', 'CN(c1ccc(F)cc1)S(=O)(=O)c1ccc(NC(=O)[C@H]2CC(=O)N(c3ccccc3)C2)cc1', 'Clc1ccc(-c2cc(Cl)cn2CCO)cc1', 'Clc1ccc(CN[C@@H](C(=O)N2CCCC2)c2ccccc2)nc1', 'ClCc1csc(Sc2ccccc2)n1', 'Clc1cc(Br)c2nc(-c3ccccc3Cl)oc2c1', 'Clc1cc(Cl)nnc1Oc1ccc(Br)cc1', 'Clc1ccccc1Oc1cc(Br)c(I)c(F)c1', 'Oc1cc(Br)cc(OCCO)c1', 'Clc1ccc(C2(Cl)CCC2)cc1', 'OCc1n[nH]c(COc2ccccc2OCc2cccc(Cl)c2)n1', 'Clc1cc(/C=Cc2ccc(COC3CCCC3)cc2)no1', 'Clc1ccc(-c2nn(-c3cc(Br)cc(F)c3)cn2)cc1', 'C[C@@H](C(=O)Nc1ccccc1F)[C@H](C)c1ccccc1', 'COc1ncncc1C(=O)Nc1ccnn1C1CCCCC1', 'OCc1nn(S(=O)(=O)c2ccccc2)c2ccccc12', 'O=C(CNc1nnc(SCc2ccccc2)o1)Nc1cccc(Cl)c1', 'CC1CCC([C@@H](C)NC(=O)Nc2ccc(Cl)cc2)CC1', 'Clc1cccc(S(=O)(=O)N2CCC(C(=O)OCc3ccsc3)CC2)c1', 'O=C(CNCc1c(C)cc([N+](=O)[O-])c2ccc(Cl)cc12)c1ccccc1Cl', 'O=C(c1cnc(N/N=C/c2ccc(Cl)cc2)o1)N1CCc2ccccc21', 'O=[N+]([O-])c1ccnc2ccc(OC(=O)C3(c4ccccc4)CCC3)cc12', 'Clc1cccc(F)c1[C@@H](O)c1cccc(OCc2ccccc2)c1', 'O=C(/C=C\\Nc1cccc(Br)c1)c1ccc(F)cc1F', 'Oc1ccc([N+](=O)c2ccc(O)cc2)cc1OCc1ccccc1', 'ClC(c1ccccc1)C1(C(=O)NC2CCCC2)CC(CC2CCCC2)C1', 'Clc1ccc(Br)cc1N1CCN(C(=O)C(F)(F)F)CC1', 'Clc1ccccc1NS(=O)(=O)c1c[nH]ccc1=O', 'Clc1ccccc1Cc1noc(Cc2ccc(Br)s2)n1', 'O=C([C@H](CC1CC1)N(Cc1ccc(Cl)cc1)C1CC1)NCC(=O)Nc1ccccc1', 'O=[N+]([O-])c1c(OCc2ccc(F)cc2)csc1NCc1ccc(F)cc1', 'Clc1ccc(-c2nonc2Cc2cccc(Br)c2)cc1', 'Oc1cc(F)ncc1COc1ccccc1F', 'Clc1cccc([C@@H](O)CNc2cccc(F)c2)c1', 'Clc1cc(Cl)nc(-c2cccnc2)n1', 'Oc1ccccc1OC[C@@H]1Cc2ccccc2O1', 'O=[N+]([O-])c1c(-c2ncnn2Cc2ccccc2)cc2ccccc2n1', 'Clc1cc(Br)c2nc(-c3ccc(Br)cc3)oc2c1', 'Clc1ccsc1NCc1ccc(F)c(S(=O)(=O)N2CCCCC2)c1', 'NS(=O)(=O)Cc1nc(Cc2ccccc2)cs1', 'O=C(c1c[nH]c2ccccc12)N1CCN(c2ccc[nH+]c2)CC1', 'Clc1ccc(Br)c(F)c1CCC(=O)NCCc1ncn[nH]1', 'O=[N+]([O-])c1ccc(CSc2nc(-c3ccc(Cl)cc3Cl)ncn2)cc1', 'Clc1ccc(-c2cc(F)c3[nH]ccc3c2)cc1F', 'Nc1[nH]nc(NCc2ccccc2)c1Cl', 'Clc1ccc([C@@H](Cl)Cc2nccn2Cc2ccccc2)cc1', 'O=C(CNc1cnn(Cc2ccccc2)c(=O)c1Cl)Oc1ccc([N+](=O)[O-])cc1', 'Oc1ccccc1-n1nnnc1SCC(=O)N1CCc2ccccc21', 'Clc1cc(C2COC2)ccc1OCc1ccc(Br)o1', 'Clc1ccc(-c2nnnn2CC2CC2)cc1F', 'Clc1ccc(-c2cccs2)c2nccnc12', 'O=C(CNS(=O)(=O)c1cc(Cl)ccc1CN1C(=O)c2ccccc2C1=O)c1ccccc1F', 'Oc1cc(F)c(Oc2ccc(Cl)cc2F)c(F)c1', 'CNS(=O)(=O)c1cc(C)ccc1NC(=O)c1c(Cl)c(C)nn1C', 'Oc1cc(Cl)cnc1SCc1ccc(F)cc1', 'Clc1ccc(Br)c(-n2nnc3cc([N+](=O)[O-])ccc32)c1', 'Clc1ccccc1OC1(NC(=O)c2ccc(Cl)cc2F)CC1', 'ClCc1cccc(C(=O)Oc2ccc3c(c2)CCCN3)c1', 'Oc1cccc2ncn(/N=C/c3ccccc3)c(=O)c12', 'Clc1cc(Cl)c(OCc2ncnn2Cc2ccccc2)c(C(F)(F)F)c1', 'ClCc1n[nH]c(=O)n1CCOc1ccccc1C(F)F', 'O=C(CNS(=O)(=O)/C=C\\c1ccc(F)cc1)Nc1ccc2c(c1)OCO2', 'O=[N+]([O-])c1ccnc2ccc([N+](=O)[O-])cc12', 'Clc1ccccc1Oc1cc(F)c(I)cc1O', 'O=C(/C=C/c1ccncc1)N/N=C/c1ccc([N+](=O)[O-])cc1', 'O=C(c1c[nH]nc1C[NH+]1CCN(C(=O)[C@@H]2CCN(c3ccccc3F)CC2)C1)N1CCCCC1', 'O=C(c1cn[nH]c1)OCC(=O)Nc1ccc(F)cn1', 'Clc1cc(F)cc(SCc2nc(Cc3ccccc3)cs2)c1', 'O=C(c1cccs1)[C@H]1CCCN(c2ccccc2)C1', 'ClCc1ncn(-c2ccccc2)n1', 'O=[N+]([O-])c1ccc(-n2cccc2)cc1O', 'Clc1ccccc1Nc1cnc(CCN2CCOCC2)[nH]1', 'O=C(c1c[nH]c([O-])n1)N(Cc1ccccc1)C[C@H](c1ccccc1)C(=O)Nc1ccc2c(c1)OCCO2', 'Cc1c(-c2cncc(NC3CCCC3)c2)oc2ccccc12', 'OCSc1nc2ccccc2[nH]1', 'Clc1cc(N2CCOc3ccccc32)nnc1-c1ccccc1', 'Clc1cccc(-c2ncoc2C2CCN(c3ccc(N4CCOCC4)cc3)CC2)n1', 'ClCc1ncnc(N2CCOCC2)c1-c1ccc(F)cc1', 'COc1ncc(CNS(=O)(=O)Cc2ccc(F)c(F)c2)cc1OC', 'Oc1ccccc1-c1c(C(=O)N(Cc2ccccc2)Cc2ccccc2)csc1C#N', 'Oc1cccc(Oc2cnnc(Nc3ccccc3)n2)c1', 'Clc1ccsc1Nc1cccc(Br)c1', 'Oc1ccc(F)c(CNCCN2C(=O)CCCC2=O)c1', 'Clc1ccc(Cc2noc(C3CC3)c2[N+](=O)[O-])cc1', 'Clc1ccccc1Oc1cc(Br)c(F)c(F)c1F', 'Oc1cccc(NC(=O)CCN2C(=O)[C@H](COc3cccc(Br)c3)C2=O)c1', 'Oc1ccc(Oc2cnccc2C2CC2)cc1F', 'OCc1nc(Cl)cc(Oc2ccc(Cl)cc2Cl)c1', 'C[C@@H](Oc1ccncc1)C(=O)Nc1nc(-c2ccccc2)ccc1F', 'O=C(c1ccsc1)c1ccc(Cl)c(C(F)(F)F)c1', 'Clc1ccccc1NC(=S)c1ccc(CCC(=O)[O-])cc1', 'Clc1ccc(-c2nonc2C2CC2)cc1', 'Clc1ccc(-c2nonc2C2CC2)cc1', 'Clc1cc(Br)cnc1OCc1cc(F)ccc1Br', 'Oc1cccc2cc(/C=C/C(=O)c3ccc(F)c(F)c3)cnc12', 'Clc1ccc(-c2cc([N+](=O)[O-])ccc2OCC(F)F)cc1F', 'O=C(COc1cc(C(F)(F)F)ccc1Cl)N(Cc1ccccc1)S(=O)(=O)c1ccccc1', 'Clc1cccc(/N=N\\c2cccc(Br)c2)c1', 'ClCc1ccn(C2CCCC2)n1', 'O=C(c1ccsc1)c1n[nH]c2ccccc12', 'O=[N+]([O-])c1ccnc2ccc(NCCNS(=O)(=O)Cc3ccccc3)cc12', 'CCN1C(=O)c2ccccc2[C@@H]1C(=O)Nc1ccc(Br)cc1', 'Clc1cc(/C=C(\\Cl)CO)ccc1OCOc1cccc(Cl)c1', 'Oc1ccc(O[C@@H]2CCN(C(=O)Cc3c[nH]c4ccc(Cl)cc34)C2)cc1', 'Clc1ccccc1Oc1cnc(C2CC2)n1C1CC1', 'O=C(CNc1cnnc2cc(Cl)c(Cl)cc12)NCc1ccco1', 'Clc1ccc(Br)c(/N=c2\\[nH]c3cnccc3n2-c2ccccc2)c1', 'Clc1ccccc1CO[C@H](c1ccco1)c1ccccc1', 'NS(=O)(=O)Cc1nnc(-c2ccccc2)n1-c1ccc(Cl)c(Cl)c1', 'COc1nc(-c2cccc(OC)c2)cc(N2CCCC2)c1[N+](=O)[O-]', 'O=C(c1ccsc1)N(Cc1ccccn1)c1nc2ccccc2c(=O)[nH]1', 'CNS(=O)(=O)c1c(F)cc(C(=O)NC2CCN(C(=O)COc3ccccc3)CC2)cc1C', 'Oc1cc(C=O)ccc1OCC(F)(F)F', 'Clc1cc(Cl)ncc1F', 'COC(=O)c1ccc(Cl)c([O-])c1C(=O)Nc1ccc(OC)cc1', 'Nc1nncn1c1ccc(Cl)cc1', 'Clc1cccc(CC[C@H](CO)Nc2ccccc2)c1', 'Oc1ccc([N+](=O)[O-])c(-n2cccc2)c1', 'Oc1cccc(Br)c1Oc1ccccc1C#N', 'Oc1ccc(O[C@@H](c2ccco2)C(=O)NCc2ccccc2)cc1', 'Clc1cc(CCl)c(F)cc1F', 'O=[N+]([O-])c1ccc(CN2CCOc3ccc(Cl)cc32)c(Cl)c1', 'CCn1nc(C)c(CNC(=O)c2cccc(-n3cccc3)c2)c1C', 'C[C@@H](Oc1ccccc1F)C(=O)Nc1cnn(CC(=O)NC2CC2)c1', 'Clc1ccccc1Oc1cc(CCCn2cccn2)ncn1', 'Oc1ccc(F)c(OS(=O)(=O)c2ccc(I)cc2)c1', 'Clc1ccccc1OCc1cscc1Br', 'Clc1ccccc1-c1csc(Cn2ccnc2)n1', 'Clc1cc(Nc2c(Cl)cccc2Cl)ncn1', 'O=C(CSCNc1ccccc1C(F)(F)F)NCc1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1N1CCOCC1', 'O=[N+]([O-])c1cc(OC2CCCC2)c2ccccc2c1F', 'Clc1ccc([C@@H](NCc2nccn2C2CCCCC2)c2ccccc2)cc1', 'NS(=O)(=O)/C(C#N)=C/c1ccc(Oc2ccc(F)cc2)cc1', 'Clc1ccc(Br)cc1Sc1nc2ccccc2cc1F', 'Clc1ccsc1S(=O)(=O)c1ccc(OC2CCCC2)cc1', 'Clc1cc(Br)cc2oc(-c3ccc(OC4CCCCC4)cc3)nc12', 'Clc1ccc(-c2n[nH]c(NCc3cccc([N+](=O)[O-])c3)n2)cc1', 'Oc1ccc(F)c(Cl)c1Cl', 'Oc1ccc(F)c(Cl)c1Cl', 'Oc1ccc(F)c(Cl)c1Cl', 'Clc1cc(Cl)c2sccc2c1Oc1ccccc1', 'Clc1cccc([S@@]c2nnc(-c3ccccc3Br)n2C2CC2)c1', 'Nc1c(Cl)nc2ccccc2c1N[C@H]1Cc2ccccc2C1', 'Clc1ccccc1Oc1cncc(Cn2ccc([N+](=O)[O-])n2)c1', 'O=C(/C(CO)(CC1CC1)c1ccc(Br)cc1)c1ccccc1', 'Clc1ccc(-c2ccccc2)cc1NC[C@H]1CCOC1', 'Clc1ccccc1-c1csc(/C=C\\c2cccc(O)c2)n1', 'ClCc1noc(-c2cnc(-c3ccc(Cl)cc3)nc2)n1', 'Nc1nonc1COc1ccc(Br)cc1', 'Oc1ccc([C@H](O)c2ccco2)c(F)c1', 'Nc1c(-c2cccc(Cl)c2)ncnc1Nc1cc(F)ccc1F', 'O=C(c1c[nH]c2ccc(Br)cc12)N1CCSCC1', 'Clc1cccc2[nH]nc(NC(=O)Cc3cccnc3)c12', 'Nc1nc(NCC2CCC2)c2c(C(N)=O)cccc2n1', 'NS(=O)(=O)CC(=O)N(CC(=O)N1CCCCC1)c1cccc(Cl)c1', 'O=C(c1cccn2c(=O)cc(C3CC3)nc12)N1CCCC1', 'CNS(=O)(=O)c1c(-c2ccccc2)nc(Nc2ccccc2)n1C', 'CCn1c(=S)sc2cc(Br)ccc21', 'O=C([O-])[C@@H]1CCCN(C(=O)Cc2c[nH]c3ccccc23)C1', 'COc1ncnc(N)c1CN1CCN(c2ccc(F)c(F)c2)CC1', 'O=C(c1cccs1)c1csc(NCc2cccc(Cl)c2)n1', 'Clc1ccccc1Cc1n[nH]c(Nc2cc(Cl)ccc2Cl)n1', 'NS(=O)(=O)/C(C#N)=C\\c1ccc(-c2ccc(Cl)cc2)cc1', 'Clc1ccccc1N1N=C(N2CCN(c3ccccc3)CC2)SC1=S', 'O=C(Cc1noc2c1CCCC2)Nc1ccc(Br)cc1', 'COc1ncnc(Nc2ccc(Cl)cc2[N+](=O)[O-])c1N', 'O=C([C@H](c1c(F)cccc1Cl)N1CCOCC1)N1CCCCC1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccn1Cc1ccc(Cl)c2ccccc12', 'NS(=O)(=O)Cc1cn(Cc2ccccc2)nc1SCc1ccccc1', 'Cc1sc2ccccc2c1CC(=O)NC1CCC(CO)CC1', 'COc1ncnc(NCc2cccc(F)c2)c1Br', 'O=C([O-])/C=C/c1ccccc1OCC1CC1', 'Oc1ccccc1Oc1ccc2c(=O)nc(NCCO)oc2c1', 'Clc1ccc(Br)c(F)c1F', 'Clc1ccc(Br)c(F)c1F', 'Clc1ccccc1Nc1cc(I)cc(CN2CCOCC2)n1', 'O=C(c1c[nH]c(-c2cccs2)n1)N(Cc1ccccc1)Cc1ccccn1', 'Clc1ccc(COCC[C@@H]2COc3ccccc32)cc1', 'Clc1cccc2[nH]nc(N)c12', 'Clc1cccc2[nH]nc(N)c12', 'NS(=O)(=O)/C=C(/Cc1ccc(O)cc1)c1ccccc1', 'Oc1cc(Cn2nc(Cl)ccc2=O)ccc1CO', 'O=[N+]([O-])c1c(F)cccc1Nc1ccc(Cl)cc1[N+](=O)[O-]', 'O=C(CN(c1cn[nH]c1)c1ccccc1)N1CCN(Cc2cccs2)CC1', 'Clc1ccc(Br)cc1NC[C@@H](O)c1ccc(Cl)cc1', 'ClCc1csc(NCc2cccc(OCC3CCOCC3)c2)n1', 'Cc1nonc1CCCN(C)C(=O)CCNc1ccccc1', 'NS(=O)(=O)c1ccc(C(=O)N2CCN(c3cc(-c4cccc(F)c4)on3)CC2)cc1', 'Clc1ccc(CC[C@@H](CBr)C2CC2)cc1', 'Clc1cc(Cl)nnc1-n1cnc2ccccc21', 'Clc1ccc(C(CO)Oc2cccc(Br)c2)cc1', 'O=C(c1ccsc1CC1CC1)N1CCCc2ccccc21', 'Clc1ccc(-c2n[nH]c(Nc3ncccn3)n2)cc1', 'Oc1ccccc1OSCOc1ccc(Br)cc1', 'ClCc1cnn(CCC(=O)c2ccc(Cl)cc2)c1-n1cccn1', 'Oc1cc(-n2nnnc2SCc2ccccc2)c2ccccc2n1', 'Clc1ccccc1NCCCSc1ncnn1C1CC1', 'O=[N+]([O-])c1c(-c2c[nH]nc2Cc2ccccc2)oc2ccccc12', 'O=C(CSCCNc1ccc2ccccc2n1)Nc1ccccc1OC(F)F', 'Oc1ccc(/C=N\\c2ccccc2Cl)c(Cl)c1', 'Oc1ccccc1C1=NN(c2ccccc2)[C@H](c2ccccc2)C1', 'Oc1ccc([N+](=O)[O-])c(Sc2cccc(Br)c2)c1', 'Nc1nonc1SCc1ccccc1OCc1ccc(Cl)cc1', 'Oc1cc(-n2cc(F)cn2)cc(C(=O)Nc2ccc(F)cc2)c1', 'Clc1nnnn1-c1ccc(O)cc1', 'NS(=O)(=O)CCN(C(=O)C1CC1)C1CCC(c2ccccc2)CC1', 'O=C([O-])/C(Cc1ccccc1)=N\\NC(=O)c1ccc(C(F)(F)F)cc1', 'O=[N+]([O-])c1ccncc1-c1ccc2c(c1)OCO2', 'O=C(/N=C/CCc1ccccc1Br)Nc1ccc2c(c1)CCC2', 'Clc1nncnc1C(=O)Nc1ccc(F)cc1', 'O=C(CNS(=O)(=O)c1cnc2c(c1)c(=O)n2Cc1ccccc1)NC1CCCCC1', 'Oc1cc(-n2cnc(-c3csc(-n4ncn(CCO)c4=O)c3)c2-c2ccccc2)ccc1O', 'O=C([C@H](c1ccc(F)cc1)N1CCOCC1)N(CCO)Cc1c[nH]c2c(Cl)cccc12', 'Cc1ccc2nc(C)c(C(=O)OCC(=O)Nc3nc4ccc(Cl)cc4s3)nc2c1', 'Clc1ccc(-c2cccs2)[nH]c1=O', 'Clc1ccc(-c2cccs2)[nH]c1=O', 'Clc1ccc(-c2cccs2)[nH]c1=O', 'Clc1ccccc1OCc1csc2ncc(-c3ccccc3Cl)n12', 'Clc1ccccc1CN(C[C@@H]1CC2(COCCO2)C1)[C@@H](c1ccccc1)C1CCCC1', 'O=C(CNC(=O)N1CCC[C@@H](c2nc3ccccc3[nH]2)C1)NC1CC1', 'O=C(CN(C1CCCCC1)c1ccccc1)N(C1CCCCC1)S(=O)(=O)c1ccccc1', 'Clc1ccc(C2CCC2)c(NCc2ccco2)c1O', 'Oc1ccc(/C=N\\N(Cc2ccccc2)S(=O)(=O)c2ccccc2)cc1F', 'Clc1ccc(-c2nnnn2-c2cc([N+](=O)[O-])ccc2OC(F)F)cc1', 'O=C(COCCO)O[C@@H](c1ccccn1)c1ccccc1', 'NS(=O)(=O)/C=C/C(=O)c1ccccc1', 'N[C@@H](Cc1c[nH]c2ccccc12)C(=O)Nc1cccc(Cl)c1', 'Clc1ccc(-c2n[nH]c(=S)n2CN2CCOCC2)cc1', 'O=[N+]([O-])c1c(Br)cccc1N1CCOCC1', 'Clc1ccccc1Oc1cnc2ccccc2c1OC[C@H]1CCCO1', 'Oc1ccccc1Oc1cccc[n+]1/N=C/c1nc2ccccc2o1', 'Clc1ccccc1Nc1cncc(C(F)(F)F)n1', 'Clc1ccccc1Nc1cnc(Cl)cc1I', 'Clc1ccc([C@@H](Cl)Cc2ccc(F)cc2)cc1Cl', 'Clc1ccc(-c2csc3[nH]ccc23)cc1', 'O=C(c1ccc(O[C@H]2CCOC2)cc1)N1CCCC1', 'O=C(c1cnc(CN2CCN(c3ncccc3F)CC2)s1)c1ccccc1', 'NS(=O)(=O)N[C@@H](c1ccccc1)c1ccc(Cl)cc1', 'Clc1ccc(C(CCF)N2CCC[C@H](c3nc4ccccc4[nH]3)C2)cc1', 'Oc1cc(F)c(/C=N/NS(=O)(=O)c2ccccc2)cc1Cl', 'O=C([C@H](Nc1ccc2c(c1)OCCO2)c1ccccc1)N(C1CCCCC1)[C@H]1CS(=O)(=O)CCS1', 'Clc1ccc(CC(=O)N[C@@H]2CCc3ccccc32)cn1', 'Clc1ccc(Br)cc1Sc1c(F)cc(F)cc1F', 'Clc1cccc(-c2ccnc(Sc3cccc(C(F)(F)F)c3)n2)c1', 'Clc1ccc(CN2CCO[C@@H](c3cccc(C(F)(F)F)c3)C2)cc1Cl', 'Clc1cc(/N=C\\c2ccccc2)cc2c1CCCC2', 'O=C(c1cc(Cl)c2c(c1)OCCO2)N1CCN(c2ncccn2)CC1', 'O=[N+]([O-])c1ccc2oc(C(=O)N3CCN(c4ccccc4[N+](=O)[O-])CC3)nc2c1', 'O[C@@H]1Cc2ccccc2CN1C(=O)COCc1ccccc1', 'Clc1nncc(-c2cccc(F)c2)n1', 'Clc1cccc2c1[C@H](c1ccccc1)N2C(=O)Cc1ccccc1C(F)(F)F', 'O=C(CSCc1csc2ccccc12)Nc1ccc2[nH]c(=O)n(-c3ccccc3)c2c1', 'O=C(/N=c1/oc2ccccc2c2ccccc12)c1ccc(NC(=O)C(F)(F)F)cc1', 'Cc1sc(=O)n(C)c1N1CCC(C(=O)NCCc2ccc(F)cc2)CC1', 'Clc1cc(Cl)c(CN(Cc2ccccc2)C(F)(F)F)cc1O', 'Clc1ccccc1CCCNC(=O)N[C@@H](C1CCCCC1)c1ccc(F)cc1', 'Clc1cc(Cl)nnc1N1CCCCC1', 'Clc1cc(Cl)nnc1N1CCCCC1', 'Clc1cc(/N=C/c2cccs2)ccc1Cl', 'O=C(/C(CCCN1C(=O)NC2(CCCCCC2)N1c1ccccc1)Oc1ccccc1)c1ccccc1Cl', 'O=[N+]([O-])c1cc(Nc2ccccc2Cl)c2ccc(F)cc2n1', 'Clc1ccc(C2=Cc3ccc([N+](=O)[O-])cc3O2)cn1', 'Clc1nnn(-c2ccccc2)c(=O)c1Br', 'ClC[C@H]1CCc2ccccc21', 'Clc1ccc(C2CN2)cc1', 'Oc1cccc(Br)c1Cn1cccn1', 'O=C(c1cc(Cl)cn2cnnc12)N1CCN(Cc2ccccc2)CC1', 'O=C(CNCc1nonc1C1CC1)N1CCC[C@H](c2nccn2C2CC2)C1', 'O=C(/N=C/CC1=Cc2ccccc2Oc2ccccc21)c1ccc(-c2cccs2)cc1', 'Clc1ccc(COc2nc(-c3cccc(Br)c3)co2)cc1', 'N=Cc1ncc(CN(C)C)cc1S(=O)(=O)Nc1ccc(Oc2ccc(C)cc2)cc1', 'Clc1ccc(Cc2csnn2)cc1', 'Clc1ccccc1C(=O)[O-]', 'Clc1ccccc1Oc1cc(Cl)ccc1C(=O)NC[C@@H]1CCOC1', 'O=C(c1c[nH]c(=O)[nH]c1=O)N(Cc1ccccc1Cl)C(F)(F)F', 'Nc1nonc1NCC(=O)Nc1cc(Cl)ccc1Cl', 'Clc1cccc2cc(-n3ccnc3)n(-c3ccc(Cl)cc3)c12', 'Clc1cc(Nc2c(Cl)cc(Cl)c(Cl)c2Cl)ccc1F', 'Oc1ccc([N+](=O)[O-])c(C(F)(F)F)c1', 'Clc1ccc(C(=O)OC[C@@H](O)c2cccc(Cl)c2)cc1', 'Clc1ccc(C2SCCCN2CCSC2CC2)cc1', 'Clc1ccc(-c2cccnc2Sc2nnc3ccccn23)cc1', 'Clc1ccc(-c2nn(-c3ccc(COCC(F)F)cc3)o2)cc1', 'Clc1cccc(/N=N/c2ccc(F)c(F)c2)c1', 'NS(=O)(=O)Cc1noc(-c2ccc(F)cc2)n1', 'Oc1ccc(/C=N\\NC2=NC=NN2)cc1', 'Clc1ccc(COc2nc[nH]c2Br)cc1', 'OC[C@H](Nc1cccc(F)c1)C(=O)N1CCCC1', 'Oc1cccc(O[C@@H]2CCOc3ccc(Oc4ccccc4F)cc32)c1', 'O=C(CNS(=O)(=O)/C=C/c1cccc(Br)c1)N1CCc2ccccc21', 'Clc1ccc(-c2cc([N+](=O)[O-])ccc2I)cc1F', 'Oc1cc(Br)c(/C=N/Nc2ccccc2)cc1F', 'O=C(c1c[nH]nc1CNCc1ccc(Br)cc1)N1CCN(c2cccc(Cl)c2)CC1', 'Oc1ccccc1[C@@H]1CN(c2ccccc2[N+](=O)[O-])CCN1CCO', 'Clc1ccc(CO/N=C/CCc2ccc3c(c2)OCO3)c(Br)c1', 'Clc1ccc(-c2n[nH]cc2CN2CCN(Cc3cccs3)CC2)cc1', 'O=C(CCS)NCc1ccc(C(F)(F)F)cn1', 'Oc1ccc([N+](=O)[O-])c(N/N=C\\c2ccccc2F)c1', 'O=C(COCCC[NH3+])Nc1ccccc1F', 'O=C(CNS(=O)(=O)c1ccc2ccccc2n1)N1CCN(S(=O)(=O)c2ccccc2)CC1', 'Clc1cccc(-c2ncnn2Cc2ccco2)c1', 'Oc1cc(CCCn2ccc(CO)n2)ccc1CO', 'CCSc1ccc(/C=C/C(=O)[O-])cc1', 'Clc1ccc(Br)c(NCc2cccnc2C2CC2)c1O', 'Clc1ccc(-c2csc3sc(=O)n(-c4ccccc4F)c23)cc1', 'Clc1ccc(Br)cc1NCC[C@H](O)c1ccccc1F', 'O=[N+]([O-])c1c(OCO)cccc1N1CCCC1', 'O=C(c1cc(Br)cs1)N1CCN(c2ccccc2C(F)(F)F)CC1', 'Clc1cccc(Br)c1C(F)(F)F', 'Clc1nncc(F)c1-c1ccc(F)cc1', 'Clc1ccccc1-c1nc(C[C@H]2CCCO2)n[nH]1', 'Oc1cccc2ncnc(Nc3cccc(I)c3)c12', 'Clc1ccc(Br)c(/N=N/c2ccccc2)c1', 'ClCc1cnc2cc(Br)ccc2c1NC1CCCCC1', 'Clc1ccc(-c2nn(-c3ccc(F)cc3)c([O-])n2)cc1', 'Clc1nncc(Sc2ccc(Cl)cc2)c1NC1CCCCC1', 'Clc1ccc(Br)cc1O[C@@H](c1ccccc1)C1CC1', 'Clc1ccccc1OCc1cc(COC(F)(F)F)cs1', 'Clc1cccc(NCCS(=O)(=O)/N=C/c2ccccc2)c1', 'O=[N+]([O-])c1c(-c2c[nH]c3ccccc23)nc2ccccn12', 'Clc1ccccc1O/C=C[C@@H](O)COCc1ccccc1', 'Clc1ccc(-c2nnnn2CCSCc2ccc(Cl)o2)cc1', 'Clc1ccc(CN[C@@H](CO)CC2CCCC2)cc1', 'O=C([C@H](Cl)c1ccccc1)N1CCOCC1', 'Clc1ccccc1Oc1cc(Oc2cc(C(F)F)ncn2)ccc1O', 'C=CCn1nc(SCC(=O)c2ccccc2)ccc1=O', 'Clc1ccc([C@@H](Br)[C@@](O)(c2ccccc2)c2ccccc2)cc1', 'Clc1cc(/N=C\\c2ccccc2Br)ccc1O', 'O=C(C1=CCN(c2ccc(C#N)c(Cl)c2)CC1)N(Cc1ccccc1)Cc1ccccc1', 'Clc1nncn1-c1cccc(Cl)c1', 'Clc1ccc(Br)c(Cl)c1Cn1cccn1', 'O=C(c1cn2ccsc2c1N)c1cc(Br)ccc1OCc1ccccc1', 'Oc1ccc(/C=N/Nc2ccc(Cl)cc2Cl)c(Br)c1', 'Clc1ccc(Br)cc1-n1cccc1Cn1ccccc1=O', 'OC[C@@H](NS(=O)(=O)c1ccc(Cl)cc1)c1cccc(F)c1', 'O=[N+]([O-])c1ccc(S(=O)(=O)NC[C@@H]2CCN(c3ccccc3)C2)cc1', 'ClCc1cc([N+](=O)[O-])ccc1Br', 'Clc1ccc(-c2n[nH]c3ccc(OC(F)(Cl)F)cc23)cc1', 'O=C([O-])/C(C(=O)c1ccccc1)NCCc1ccccc1', 'O=C(c1cc(F)cc2cc[nH]c12)N1CCO[C@@H](c2ccccc2)C1', 'O=C(/C=C\\c1cccs1)NC1CCN(c2ncccn2)CC1', 'Clc1nnn(-c2cccc(Cl)c2)c1Cc1ccccc1Cl', 'Clc1ccccc1NCC1CCN(S(=O)(=O)c2ccc(Br)o2)CC1', 'Clc1cc(Br)c(Sc2cccc(F)c2)nc1', 'O=C(CNS(=O)(=O)/N=C/c1ccccc1)NCCc1ccc(Cl)cc1', 'CNC(=O)Cn1ccc(=O)n(C)c1=O', 'Clc1cccc2nn(CCCCNc3cccc(Cl)c3)nc12', 'Clc1ccccc1-c1noc(/C=C/c2ccc(F)cc2Cl)n1', 'Clc1ccccc1[C@H](Cl)C1CCCC1', 'Clc1ccccc1CSc1nnc(SCc2cccc3ccccc23)n1C1CCCC1', 'ClCc1csc(CN2CCN(c3ccc(Oc4ccccc4)nn3)CC2)c1', 'Clc1ccccc1-c1cc(C2CCOCC2)ncn1', 'Clc1cccc2c1Oc1nc3c(O)c(O)ccc3nc1n2Cc1ccco1', 'Clc1ccccc1OCCOCCOC[C@@H]1CCCO1', 'Clc1cccc(/C=C2\\SC(=S)NC2=O)c1', 'Clc1ccc(Br)c(S(=O)(=O)N2CCSCC2)c1', 'Clc1nncc(-c2ccc(Cl)c(Cl)c2)n1', 'O=[N+]([O-])c1cc(N2CC(C(=O)NCc3cccs3)C2)ccc1Br', 'Clc1ccc(Cc2nnn(CC(=O)NCCN3CCOCC3)c2Cl)cn1', 'Clc1ccccc1NCC[C@H]1CCCN1S(=O)(=O)c1c[nH]c2ccccc12', 'Clc1cccc2ccc(/C=C/O)cc12', 'O=C(Cc1nonc1C(F)(F)F)Nc1ccc(Cl)cc1', 'COc1ncccc1COC(=O)c1cccc2cc[nH]c12', 'Clc1nncn1Cc1cnn(-c2ccccc2)c1', 'O=C(CSCC1(O)CCOCC1)c1cccc(F)c1', 'O=C(c1c[nH]nc1-c1ccc(F)cc1)c1cc(S(=O)(=O)N2CCCCC2)ccc1O', 'Clc1ccc(-c2cc[nH]n2)c2ccccc12', 'Oc1ccc(F)c(Cl)c1COc1cnnc2ccccc12', 'Cc1cnc2[nH]c(C(=O)Nc3ccc4c(c3)OCO4)cc2c1', 'Clc1ccsc1/CCCn1ccc(=O)c2ccccc21', 'NS(=O)(=O)/N=C(\\NCc1ccc(F)cc1)C1CCCCC1', 'C[C@@H](NS(=O)(=O)c1ccc2ccccc2c1F)C(=O)NCCCc1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(-c2ccc(F)cc2)c1N', 'Clc1cccc(F)c1N/C=C/c1ccc([N+](=O)[O-])cc1', 'Clc1ccc(-c2cc(Br)cnc2Cn2ccnc2)cc1', 'Oc1cccc(Br)c1OCC1(O)CCCCC1', 'Clc1cccc(Br)c1CNc1cc([N+](=O)[O-])ccc1O', 'Oc1cccc(-n2c(Sc3ccccc3)n[nH]c2=O)c1', 'Nc1cnnn1-c1ccccc1/C=C/C(=O)c1ccc(Br)cc1', 'Nc1c(C(=O)N[C@H](CO)c2ccc(Cl)cc2)cnn1-c1ccccc1', 'Clc1cc(Br)c(S(=O)(=O)N2CCCC2)cn1', 'Nc1cn[nH]c1-c1ccccc1Br', 'Clc1ccc(C(F)(F)F)c(F)c1Cl', 'Clc1ccc(Br)cc1S[C@H]1Cc2ccccc2C1', 'Clc1cc(CCc2c[nH]c3ccccc23)[nH]c1', 'Oc1cc(-c2noc(C3CC3)n2)ccc1SCC(=O)Nc1cccc(Cl)c1', 'Cc1sc(NCCN(C)C)nc1S(=O)(=O)N1CCN(c2ccc(F)cc2)CC1', 'Clc1ccccc1NC(=O)[C@H](Cc1c(Cl)cccc1Cl)Nc1ccccc1O', 'ClCc1cc(F)ccc1-n1cncn1', 'Clc1ccc(C(=O)Nc2ccc(Oc3c(F)cccc3F)o2)cc1', 'Clc1ccc(Br)cc1-n1nnnc1SCc1ncc(-c2ccccc2)o1', 'Clc1cc(Cl)c(Cn2c(=O)[nH]c3ccccc32)c(Br)c1Br', 'Clc1nncc(OCc2ccccc2F)n1', 'Clc1cc(N2CCOCC2)nc(-c2ccco2)n1', 'Oc1cccc(SCn2cnc3ccccc3c2=O)c1', 'O=[N+]([O-])c1c(F)ccc(CNCc2nc(-c3ccncc3)no2)c1Cl', 'Nc1[nH]ncc1Nc1ncnc(Nc2ccc(Cl)cc2)n1', 'CNC(=O)c1cccc([C@@H]2CCCCN2C(=O)c2cc(F)ccc2)c1', 'Clc1cc(F)c(C(F)(F)F)cc1OC1CCC1', 'O=C(c1ccccc1Sc1ccccn1)N1CCN(c2nccs2)CC1', 'Oc1ccccc1Oc1c(Cl)ccc2c1NCCN2C(=O)c1ccccc1', 'Clc1ccccc1-c1n[nH]c2cn[nH]c(=O)c12', 'Clc1ccc(Cc2c[nH]nc2-c2ccccc2)c(Cl)c1', 'Clc1ccc(-c2nonc2N2CC(CO)C2)cc1', 'Oc1ccccc1OC[C@H](O)c1ccc(Cl)c(Cl)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(Br)cc1N1CCN(c2ncccn2)CC1', 'O=[N+]([O-])c1ccc2nc(-c3cccc([N+](=O)[O-])c3)[nH]c(=O)c2c1', 'Clc1ccccc1Nc1cc(OCc2nc3ccccc3s2)cs1', 'ClC[C@H](Br)Cc1ccccc1', 'Clc1nnnn1-c1cccc(Cl)c1', 'O=[N+]([O-])c1cc(F)c(Br)cc1Cl', 'Clc1cc(Cl)cc2[nH]ncc12', 'Oc1cc(OCOCC2CCN(c3ccccc3)CC2)ccn1', 'Oc1cc(C(F)(F)F)cc2c1ncn2-c1ccc(F)cc1', 'Clc1ccsc1CN(Cc1c[nH]c2ccccc12)C1CC1', 'N#CCSc1ccccc1Cn1c(=O)[nH]c2ccccc21', 'NS(=O)(=O)c1ccc(S(=O)(=O)C[C@H](O)c2ccccc2)cc1', 'Clc1ccccc1Oc1cnc2cc([C@H]3CCCN(C(=O)c4ccccc4)C3)[nH]c2c1', 'O=[N+]([O-])c1c(F)cc(NCc2cccc(O)c2)cc1Cl', 'Clc1nnn(-c2ccccc2)n1', 'O=C(COc1ccc(I)cc1)Nc1ccnc(C(=O)[O-])c1', 'Oc1ccccc1O[C@@H](c1ccccc1)c1cccc(F)c1', 'O=C(CSCC(F)(F)F)NNC(=O)c1ccc(S(=O)(=O)N2CCCC2)cc1', 'Oc1cc(C=O)nc(-c2ccccc2)n1', 'N=Cc1ccc([C@@H](C)NC(=O)C2CC2)cc1', 'O=C(C1CCCC1)N1CCS(=O)(=O)c2ccccc21', 'O=C(c1cccs1)C(=O)NCc1ncc(Cc2ccccc2)o1', 'O=C(c1cccc(-n2cccc2)c1)N1CCC(Oc2cccnc2Cl)CC1', 'O=C(/C=C\\c1nc(-c2ccc(Cl)s2)cs1)c1ccccc1', 'Clc1ccc(-c2cc(Cl)cs2)cc1F', 'Clc1cccc(C2(CO)CC2)c1Cl', 'Clc1ccc(-c2cc(Sc3nc4ccccc4[nH]3)co2)cc1', 'O=C(c1c[nH]c2ncnc(NC3CC3)c12)N1CCOCC1', 'Clc1ccc(Br)cc1[C@@H](Cl)c1ccccc1', 'C=C[C@@H](NC(=O)c1ccc(F)cc1)c1ccccc1OC', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1-c1ccc(Cl)cc1', 'O=C([C@H](SCO)c1ccc(Br)cc1)N1CCN(Cc2ccccc2)CC1', 'C=CC#CCc1cccc(NC(=O)N2CCC(NC(=O)c3ccccc3)CC2)c1', 'CCc1ccc2nc(N(Cc3ccco3)C(=O)c3ccc(O)c(F)c3)[nH]c2c1', 'Oc1cccc(OC[C@H]2CCCO2)c1', 'Oc1cc(OCCn2cnc(-c3ccccc3)n2)cc(Cl)c1OCc1cccs1', 'C=C[C@@H](Oc1ccccc1OC)c1ccc(F)cc1', 'O[C@@H](CCc1ccccc1)C(=O)Nc1ccccc1C(=O)[O-]', 'Oc1ccccc1[C@@H](Br)[C@@H](CO)Cc1ccccc1', 'O=[N+]([O-])c1cc(Br)c(-c2cccc(Br)c2)cc1O', 'O=C(CS(=O)(=O)c1ccccc1)NCC1(C2CC2)CCCCC1', 'O=C(Cc1conn1)Nc1ccc2ccccc2c1', 'O=[N+]([O-])c1c(Cl)ncc2ccc(Cl)cc12', 'Oc1cc(O)cc(/C=C(/C(=O)NCCc2ccccc2Cl)C(F)(F)F)c1', 'Clc1cccc2c1CC[C@@H]2c1ccc(Cl)cc1', 'Nc1[nH]ncc1N1CCN(C(=O)c2c(Cl)cccc2Cl)CC1', 'O=C([C@H](Sc1ccc(Cl)cc1)C(=O)NCc1ccccc1)NC1CCCC1', 'O=C(c1ccc[nH]c1=O)Oc1ccc(Br)c([N+](=O)[O-])c1', 'O=C(CNc1nn2cnnc2c2c1CCCC2)NNC(=O)c1ccc(Br)cc1', 'Oc1ccc(O[C@H](C(=O)NNC(=O)c2nc(C3CC3)c3ccccn23)c2ccccc2)cc1', 'Cc1ccsc1OCC(=O)Nc1cccc(C(=O)N2CCOCC2)c1', 'ClCc1csc(N2CCOCC2)n1', 'Clc1ccc([C@@H](NS(=O)(=O)c2ccc(Cl)cc2)N2CCCC2)cc1', 'O=C(Cc1n[nH]c(=O)[nH]1)Nc1ccc2c(c1)oc(=O)n2Cc1ccccc1', 'Clc1ccccc1Oc1cnc(CSc2ncccn2)nc1', 'NS(=O)(=O)Cc1csc(NC(=O)c2cccc([N+](=O)[O-])c2)n1', 'Clc1cccc2c(Oc3cccc(Cl)c3)n[nH]c12', 'O=C(CSCC1CCCCC1)N1CCC[C@@H](c2nc3ccccc3n2C2CC2)C1', 'Clc1ccc(Cc2noc(Cc3cccs3)n2)cn1', 'Clc1ccccc1NC(=O)[C@H]1COc2ccccc2O1', 'Clc1cccc(/C(Br)=C/CN2CCOCC2)c1', 'OCc1ncsc1Nc1ccccc1C(=O)N1CCCC1', 'Clc1ccc(Br)cc1OS(=O)(=O)CCO', 'Clc1ccccc1-c1csc(Sc2cccnc2)n1', 'CNc1nnc(SCC(=O)Nc2cccc(Br)c2)nc1N', 'Oc1cc(F)ncc1Cn1cnc2ccccc21', 'Clc1ccccc1[C@H](c1ccccc1)Sc1nnnn1-c1ccccc1', 'Clc1ccc(Cc2cc(F)c(F)c(Cl)c2)cc1Br', 'Oc1cc(C(F)(F)CN2CCOCC2)cc(Cl)c1OCc1ccccc1', 'O=[N+]([O-])c1ccnc2ccc(Cl)c(F)c12', 'CCCCc1ccc(CNC(=O)NC(C)(C)c2cccc(C)c2)s1', 'ClCc1cc(F)cc(N2CCNCC2)c1', 'Clc1ccc(-c2ccccc2Cl)cc1NCC(=O)[O-]', 'Clc1nnnc(NNc2ccc(S(=O)(=O)N3CCCCC3)cc2)c1Cl', 'Clc1cc(Br)c2nc(-c3ccncc3)ncc2c1', 'Oc1cccc(Nc2cccc(F)c2Br)c1O', 'Clc1ccccc1-c1nn(-c2ccc(Br)cc2)cc1[O-]', 'O=C(/N=C\\c1cc2ccccc2[nH]1)c1cccc(Cl)c1', 'Oc1cccc(SC2=[N+](c3ccccc3)Nc3cc(F)c(F)cc32)c1', 'ClCc1ccc2nc[nH]c(=O)c2c1', 'Oc1ccc(/C=C/N2CCOCC2)cc1', 'COCc1c([O-])n2cc(-c3ccccc3)cc(-c3ccc(Cl)cc3)c2n1', 'O=C(Cc1nnc(-c2ccc(Br)cc2)o1)N[C@@H](c1cscc1)c1ccccc1', 'O=C(c1c[nH]cn1)N1CCC(Oc2ccc(C(=O)NCc3cccs3)cc2)CC1', 'Clc1ccccc1-c1n[nH]cc1-c1ccc(Cl)cc1Cl', 'Oc1ccccc1-c1c(Cl)cc(Cl)cc1CBr', 'Nc1n[nH]c2cc([N+](=O)[O-])ccc12', 'Clc1ccsc1/N=C/c1cccc(-c2c(F)cccc2F)c1', 'Clc1ccccc1COCc1ccc(OC[C@H]2CCCO2)cc1Cl', 'Oc1cc(CCOc2ccccc2F)c2cccnc2n1', 'Clc1ccc(C(=O)COC[C@@H]2CCOC2)cc1', 'C=C1c2ccc(OCC(=O)OCC(=O)N/N=C/c3ccccc3)cc2O1', 'Cc1cncc(NC(=O)c2nnc(-c3ccc4ccccc4n3)s2)c1', 'Clc1nncn1c1ccc(Cl)c(Cl)c1', 'O=C(/N=C/NCc1ccccc1)c1ccc(F)cn1', 'O=C(Cc1cnn(-c2ccc3c(c2)OCO3)c1C#N)Nc1ccc2c(c1)OCO2', 'Nc1[nH]nc(SCCOc2ccc(F)cc2)c1[N+](=O)[O-]', 'Clc1nnn(Cc2ccccc2)c1I', 'Clc1ccc(-c2cccnc2S(=O)(=O)C(F)(F)F)cc1', 'OC1=NC(SCC(=O)Nc2cccc(Cl)c2)=NN1', 'Clc1ccc(Br)c(-n2nnnc2SCc2csc(-c3ccccc3)n2)c1', 'CCn1ncc2cccc(CC(=O)NCc3ccccn3)c2c1=O', 'O=C(c1ccsc1)c1nnc(-c2ccccc2)s1', 'Clc1nnnn1c1cccc(OCCO)c1', 'Clc1ccc(CN[C@@H](CCO)C2CC2)cc1', 'Clc1cc(Br)c(NCc2ccncc2)nc1Cl', 'O=[N+]([O-])c1c(OCOc2cccc(Cl)c2Cl)cccc1[N+](=O)[O-]', 'Clc1cc(/N=C/c2nnc3ccccn23)cc(OCc2ccccc2)c1Cl', 'O=C(CS(=O)(=O)CCN1C(=O)/C(=C/c2ccc(C#N)cc2)SC1=S)Nc1ccccc1', 'Cc1nonc1CN(Cc1ccccc1)c1ncccn1', 'N#C/C(=C\\c1ccc(-c2cccc([N+](=O)[O-])c2)o1)C(=O)NCCc1ccccc1', 'CN=C(c1ccccc1)c1ncn(-c2ccccc2)c1[N+](=O)[O-]', 'Clc1cc(F)cc(N[C@H]2CCN(c3ccccc3)C2)c1', 'Clc1ccc(-c2nnnn2/N=C/c2cccc([N+](=O)[O-])c2)cc1', 'Clc1ccc(-c2nnnn2/N=C/c2cccc([N+](=O)[O-])c2)cc1', 'Clc1ccc(Br)cc1Sc1nc2ccc(F)cc2n1C1CC1', 'O=C(c1ccsc1)c1nn(Cc2ccc(F)cc2)c2c1CCCC2', 'Cc1c([N+](=O)[O-])cc(C#N)c(C)c1SCC(=O)NCc1ccccc1', 'Clc1ccccc1-c1cscc1C(F)(F)F', 'Clc1ccccc1-c1cscc1C(F)(F)F', 'Clc1ccccc1-c1cscc1C(F)(F)F', 'Oc1cccc2cc(/C=N\\c3ccc(Cl)cn3)ccc12', 'Clc1cccc(Nc2cc(-c3cccs3)no2)c1', 'Clc1nnnn1c1ccc(CSc2nnc3ccccn23)cc1', 'Clc1ccc(CCN2CCN(c3cccc[nH+]3)CC2)cc1', 'N[C@@H](CCSCc1ccco1)C(=O)Nc1ccccc1', 'O=C(c1cc(Br)c[nH]1)c1cc(F)ccc1OCCN1CCCC1=O', 'O[C@@H](CNc1cnccn1)c1ccc(Cl)cc1', 'O=C([C@H](Cl)c1ccccc1F)N1CCN(c2ccccc2)CC1', 'Clc1cc(Cl)nnc1N/N=C/c1ccc(Cl)cc1', 'O=C(c1cc(Br)c[nH]1)C1CCCCC1', 'O=C(COc1cc(N2CCCCC2)nn1-c1ccccc1F)N1CCN(S(=O)(=O)c2cccs2)CC1', 'O=C(/C(c1cccs1)=C/c1ccc(O)cc1)c1ccc(S(=O)(=O)N2CCCC2)cc1', 'Oc1ccc(O[C@@H]2C[C@@H](c3cccc(COC(F)(F)F)c3)C2)cc1', 'Nc1cn[nH]c1C(=O)CCCCS(=O)(=O)c1ccc(F)cc1', 'O=[N+]([O-])c1cc(Br)ccc1Sc1ccc(F)cc1O', 'Clc1cccc([S@@]C(CC2CC2)C2CC2)c1', 'O=C(/C=C1CCN(C(=O)CCc2ccccc2)N=N1)Nc1ccccc1F', 'Clc1cc(Cl)cnc1-c1ccc2c(c1)COCO2', 'O=[N+]([O-])c1ccncc1F\n\n', 'O=[N+]([O-])c1ccncc1F\n', 'O=[N+]([O-])c1ccncc1F', 'O=[N+]([O-])c1ccncc1F', 'Clc1ccc([C@@H](CNCc2ccccc2)N2CCCCC2)c(Br)c1', 'Clc1cc(/C=N/Oc2ccccc2Cl)ccc1OS(=O)(=O)c1ccccc1', 'CN(Cc1ccc2ccccc2c1)C(=O)CN1C(=O)N[C@](CNS(=O)(=O)c2ccccc2)C1', 'NS(=O)(=O)Cc1nnc(-c2ccc(N3CCCC3)cc2)n1N', 'Nc1[nH]nc(N(CCc2ccccc2)C2CCCCC2)c1[N+](=O)[O-]', 'Clc1cccc(Nc2cc(Br)ccc2OC[C@H]2CCCO2)c1', 'Clc1ccc(-c2cc(/C=C/c3cccnc3)n[nH]2)cc1', 'Clc1ccc(Br)c(/C=N/c2c[nH]c3ccccc23)c1', 'Clc1cccc([S@@]c2nc3c(n2C)CCCC3)c1', 'Oc1cc(Br)c(N/N=C\\c2ccccc2)cc1Cn1ccnc1', 'Clc1ccsc1S(=O)(=O)Nc1ccc(C(F)(F)F)cc1', 'Clc1cc(C(=O)NCCCc2ccsc2)cs1', 'Clc1ccccc1Oc1cc(Br)ccc1C(=O)[O-]', 'O=[N+]([O-])c1ccncc1OCCC(F)(F)F', 'Clc1ccc(C(=O)N2CCCC2)c2c1cnn2-c1ccc(Cl)cc1Cl', 'O=[N+]([O-])c1ccc(-c2n[nH]cc2CC(=O)N2CCCC2)cc1', 'Clc1nncc(F)c1-c1ccc(Cn2ccccc2=O)cc1', 'Clc1ccc(Br)cc1N(C1CCOCC1)C1CCCCC1', 'Clc1ccc([C@@H](O)[C@@H]2CC(c3ccc(O)cc3)c3ccccc32)cc1', 'Oc1cc(C(F)(F)F)nc(CN(Cc2ccccc2)C2CC2)c1', 'Oc1cccc2nnc(-c3ccccc3F)n12', 'CNC(=O)CNC(=O)C1CCN(c2ncnc3[nH]cnc23)CC1', 'Clc1ccc(Cc2noc(CCC3CCCCC3)n2)o1', 'Oc1cc(F)ncc1S(=O)(=O)c1ccc(F)cc1', 'Clc1cc(Br)c2nc(-c3cc(F)ccc3Cl)oc2c1', 'Clc1ccccc1-c1cscc1-n1cccn1', 'Clc1ccc(-c2cc(O)n[nH]2)cc1', 'Oc1ccc(F)c(S(=O)(=O)c2cccc(F)c2F)c1', 'Clc1ccccc1Oc1cnc2sn(-c3ccccc3)c2c1', 'Oc1ccc([N+](=O)c2ccccc2)c(Oc2cccc(F)c2)c1', 'Clc1ccccc1[C@H]1CN(C(=O)Cn2nnnc2-c2ccccc2)CCO1', 'O=[N+]([O-])c1cccc2[nH]cnc12', 'Clc1cc(N2CC[C@H](Cc3ccccc3)C2)nc2ccccc12', 'O=[N+]([O-])c1ccn(Cn2nnnc2-c2ccccc2)c1', 'Clc1ccccc1OCc1ccn(Cl)n1', 'Oc1cccc(Oc2cccc(COc3cnccn3)n2)c1', 'O=[N+]([O-])c1c(F)ccc2c1N=C(c1ccccc1)OC2=O', 'Oc1cccc(-c2cnc3cc(Cl)ccn3c2=O)c1', 'O=C(CCC1=CCCC1)N[C@H](Cc1ccccc1OC(F)F)c1ccccc1', 'CNS(=O)(=O)Cc1ccc(F)c(C(=O)Nc2cccc(F)c2)c1OC', 'Clc1cc(/C=C(\\C#N)c2cccc(Cl)c2)cc(Cl)c1Cl', 'OCc1n[nH]c(Nc2cccc3ccccc23)n1', 'Oc1ccc([N+](=O)Cc2ccccc2)cc1Cl', 'Nc1noc2cc(CCSc3ccc(O)cc3)ccc12', 'Oc1cc(Br)cc([C@H](c2ccccc2)N2CCOCC2)c1', 'Clc1ccccc1Oc1cc(-c2ccncc2)no1', 'O=C([C@H](c1ccc(Br)cc1)N1CCN(Cc2c[nH]c[nH+]2)CC1)c1ccccc1F', 'O=C(c1cccc(Cc2nnc(-c3ccccc3)o2)c1)N1CCOC(N2CCOCC2)C1', 'ClCc1ccnn1-c1ccc([N+](=O)[O-])cc1', 'Oc1cccc2cc(OC[C@H]3CCCO3)ccc12', 'Oc1ccc([N+](=O)[O-])c(Sc2cc(F)ccc2F)c1', 'Clc1ccc(-c2n[nH]cc2N2CCN(c3ncccn3)CC2)cc1', 'Clc1ccc(-c2nn(-c3cc(F)ccc3F)c(=S)s2)cc1', 'COc1nnnc(NC(=O)Cc2ccc(I)cc2)n1', 'C[C@@H](C(=O)Nc1ccc(N2CCCCC2=O)cc1)n1cc(-c2ccc3ccccc3c2)nn1', 'ClCc1cn[nH]c1-c1ccccc1C(=O)Nc1ccc([N+](=O)[O-])cc1Cl', 'C=CCCn1/c(=N/C(=O)c2ccccc2)sc2cc(C)c(C)cc21', 'Clc1ccc(-c2nn(-c3ccc(S(=O)(=O)C4CCCCC4)cc3)o2)cc1', 'O=C(c1cc(F)ccc1F)N(CCCN1CCOCC1)S(=O)(=O)c1cccc(Cl)c1', 'Clc1cccc2cc(Br)sc12', 'Clc1cccc2cc(Br)sc12', 'O=C(CSCCOc1ccccc1-c1ccccc1)NNC(=O)c1cnc2ccccc2n1', 'Clc1ccc(Br)cc1OS(=O)(=O)C(F)(F)F', 'Oc1ccc(O[C@H](CSc2ccccc2)C(=O)NC2CC2)cc1', 'CN(Cc1ccc([O-])cc1)C(=O)c1ccc(Cl)c(N2CCCCC2)c1', 'Clc1ccc(Br)cc1NC(=S)NCc1ncc[nH]1', 'O=C(/N=C\\c1c[nH]c2ccccc12)c1ccc([N+](=O)[O-])cc1', 'Clc1ccc(Br)cc1OCC[C@H]1COc2ccccc2O1', 'OC[C@H](CO)Nc1ncnc2c1c(=O)n(C)c1ccccc12', 'Clc1ccc(-c2csc3sc4c(c23)CCCC4)cc1', 'Clc1ccccc1-c1nncc(Cl)c1F', 'O=[N+]([O-])c1ccncc1COc1cccc2c1C(=O)CCC2', 'Clc1ccc(-c2csc3sccc3c2=O)cc1', 'Oc1ccc(F)cc1-c1nnc(CNCc2ccc(C(F)(F)F)cc2F)o1', 'ClC[C@H](c1c[nH]c2ccccc12)N1c2ccccc2C[C@@H]1Cc1cccs1', 'Oc1ccc(/C=C(\\Cl)CS(=O)(=O)c2cccc(Cl)c2)cc1', 'CC(C)[C@H](C(=O)Nc1ccccc1)N(C)C(=O)c1ccc2c(c1)OCCO2', 'O=C(Cc1nn[nH]c1C(=O)c1ccc2c(c1)CCCC2)c1ccccc1', 'Oc1ccc(/C=C2/N=C(CN3CCN(c4ccccc4F)CC3)N=N2)cc1', 'ClCc1nnn(-c2ccccc2)n1', 'O=C(c1cn(C[C@@H]2CCCO2)cn1)c1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1C(=O)Nc1csc2ccccc12', 'Clc1ccc(Br)cc1NCC1(C2CC2)CC1', 'Clc1cc(CCN[C@H]2CCCc3ccccc32)ccn1', 'O=C(c1cn2ccsc2n1)C1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1', 'O=C(c1cnn(-c2ccccc2Cl)c1)N1CCC(C(F)(F)F)CC1', 'O=C(c1cc(F)cc(F)c1)N1CCC(n2cncn2)CC1', 'Clc1cc(Cl)nc(N2CCC(CSc3ccc(O)cc3)CC2)c1', 'Oc1ccccc1[C@@H](CO)Nc1ccccc1OCc1cccs1', 'Oc1cc(COc2nc(-c3ccc(Cl)cc3)nc3c2CCC3)ccn1', 'Clc1ccc(-c2nnnn2Cl)cc1', 'Clc1ccc(-c2nnnn2Cl)cc1', 'Clc1ccc(-c2nnnn2Cl)cc1', 'Clc1ccc(-c2nnnn2Cl)cc1', 'Clc1ccc(-c2nnnn2Cl)cc1', 'COc1nn(-c2c(Cl)cccc2Cl)c(-n2nc(C)cc2C)c1NC(=O)c1ccc(NC(C)=O)cc1', 'Clc1ccc(-c2nnnn2/N=C/c2ccc(Cl)cc2Cl)cc1', 'Clc1ccccc1NC(=O)[C@@H](CO)CCl', 'O=C(c1c[nH]c2nc[nH]c12)N1CCN(c2ccc(F)cc2)CC1', 'CNC(=O)N1CCC(CCNC(=O)NCc2nc3cc(C)ccc3n2CC)CC1', 'Clc1ccc(Br)c(-n2nc(COC(=O)C3CCCC3)cc2Cl)c1', 'COc1ncncc1CSCOC(=O)Cc1ccccc1', 'O[C@@H](Sc1ccccc1)C(=O)N1CCCCC1', 'Clc1cccc2ncn(CCCSc3ccccn3)c12', 'Clc1ccccc1Oc1cnc2sc(C3CC3)nc2c1', 'O=C(COCCCNC(=O)C1(c2ccccc2)OC1=O)c1ccccc1', 'Oc1ccccc1OSCc1noc(-c2ccccc2)n1', 'ClCc1cnn(-c2cccc(Cl)c2)c1Cl', 'N#Cc1c(SCCC(=O)Nc2ccc(Br)cc2)n[nH]c1NC(C)C', 'C=C1c2ccc(OC)c(C(=O)NCc3ccccc3)c2[C@H]1c1ccc(SC)cc1', 'O=C(Cc1coc(-c2cccs2)n1)N1CCN(C[C@H]2CCCO2)CC1', 'COCc1c(-c2ccccc2)[nH]c(=S)n1C', 'Oc1ccc(F)cc1OC[C@H](O)c1ccc(Cl)cc1', 'Clc1ccc(-c2n[nH]cc2NC(=O)c2cc([N+](=O)[O-])ccc2Cl)cc1', 'Oc1ccccc1CO[C@@H]1CCCC[C@@H]1[C@@H]1CCCN1C(=O)c1ccccc1', 'Oc1cc(Br)c(-c2ccncc2)c(Br)c1', 'N[C@@H](c1ccc(F)cc1)c1c(F)cccc1Cl', 'Clc1ccc(Cc2noc(-c3ccco3)c2Cl)cc1', 'Clc1cccc(/N=N\\NC(=S)NNC(=O)Cc2ccccc2)c1', 'O=C(/C(Cc1ccccc1)Nc1nccs1)c1ccc(F)cc1', 'Clc1ccc(CC[NH2+]Cc2ccccc2)cc1', 'NS(=O)(=O)Nc1cc(-c2ccco2)nc2ccc(Cl)cc12', 'NS(=O)(=O)CCc1nc(-c2ccccn2)no1', 'O=C(C1CCN(c2ccc(F)cc2F)CC1)C1CCC(c2cn[nH]c2)CC1', 'ClCc1ccn(C[C@H](O)c2ccccc2)c1', 'ClCc1ccn(C[C@H](O)c2ccccc2)c1', 'Oc1ccccc1CCN1CCC([NH2+]Cc2ccc(CO)cc2)CC1', 'Clc1ccc(Br)c(SC2CCCC2)n1', 'C[C@@H](c1ccccc1)N(C)C(=O)Cc1c[nH]c2ncccc12', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1NCCN1C(=O)N[C@](CC2CC2)C1', 'C[C@@H](NS(=O)(=O)c1ccccc1[N+](=O)[O-])c1ccc(F)cc1F', 'Clc1cc(Br)cn1CCNC(=O)C1CCN(S(=O)(=O)c2c(F)c(F)c(F)c(F)c2F)CC1', 'Clc1ccccc1Nc1cnc(I)c(Br)c1', 'Clc1ccccc1-c1nc(CN2CCOCC2)nc2nc[nH]c12', 'Clc1ccc(-c2n[nH]cc2CC2CC2)cc1', 'ClCc1ccn(-c2ccc(C3CCCCC3)nn2)n1', 'O=C(COCCC[C@@H]1COc2ccccc2O1)NCc1ccc2c(c1)CCCC2', 'Oc1cc(-n2nccc2C(=O)c2ccccc2OCc2ccc(Cl)cc2Cl)cc2c1NCCC2', 'Clc1ccccc1N1N=C(C(=O)Nc2ccc([N+](=O)[O-])cc2)SC1=S', 'Clc1cccc([S@@]C2CCN(S(=O)(=O)c3cccnc3)CC2)c1', 'Clc1ccccc1NC(=O)CSc1nnc(-c2ccccc2)n1C[C@@H]1CCCO1', 'Clc1ccc(Br)cc1N1C[C@H](c2ccccc2)CC1', 'O=C(/C=C\\CN1CCSC1=O)c1ccc(Cl)c(Cl)c1', 'Oc1cccc(OC[C@@H]2CO2)c1', 'O=C(Cc1nn(CC[NH+]2CCCCC2)c(=O)c2ccccc12)N1CCOCC1', 'COc1nn(-c2c(C)cccc2F)c2cccnc12', 'Clc1ccc(-c2csc3[nH]c4ccccc4c(=O)c23)cn1', 'Cc1c(Br)onc1CCN(C)C(=O)NCc1ccccc1', 'O=C(c1cc(Br)cnc1Br)N1CCN(C(=O)c2cccs2)CC1', 'Clc1ccc(-c2nonc2)cc1', 'Clc1ccc(-c2nonc2)cc1', 'Clc1ccc(-c2nonc2)cc1', 'OCc1n[nH]c(Nc2cccc(-c3ccccc3)c2)c1C1CC1', 'O=C(/C=C\\N1CCC(OC2CCCCC2)CC1)c1ccccc1F', 'O=C(CSCNS(=O)(=O)c1ccc(Cl)cc1)NNC(=O)OCc1ccccc1', 'O=C(CCc1c[nH]c2ccccc12)N[C@H]1CCC[C@H](O)C1', 'Clc1ccccc1NCc1cc2ccccc2n1C[C@@H]1COCCO1', 'Clc1ccc([C@@H](CC(=O)NCCN2CCCC2)C(F)(F)F)cc1', 'Clc1ccccc1O/Cn1c(=S)oc2ccc(Cl)cc21', 'C[C@@H](Oc1ccccc1)C(=O)Nc1ccc([N+](=O)[O-])c([N+](=O)[O-])c1', 'O=C(/N=C/C=C\\c1cccc2c1OCO2)c1ccccc1', 'Oc1ccc(F)c(-c2noc(-c3ccsc3)n2)c1', 'Clc1ccc(-c2csc3sc4c(c23)CCC4)cc1', 'CCn1nc(CNC(=O)CC(C)C)c(=O)c(C)c1C', 'Clc1cc(Nc2ncc(I)cc2F)ccc1O', 'Clc1ccc(Br)cc1/N=N\\c1ccc(Br)cc1', 'O=C(/C=C/c1c(NC2CC2)oc2ccccc12)NNC1CCCC1', 'Clc1ccccc1Cn1nc2cc(-c3ccccn3)[nH]nc2c1=O', 'Clc1ccc(-c2cccs2)cc1OCCC1(F)CCCC1', 'Oc1cccc(CCl)c1F', 'OCc1nncn1Cc1cccc(F)c1F', 'N#C/C(=C\\c1cccc(NC(=O)c2cnn(C3CCC3)c2)c1)C(N)=O', 'Clc1cc(F)cc(N2CCCC[C@@H]2Cc2ccccc2)c1', 'COCc1nc(C)ccc1C(=O)O[C@@H](CC)c1ccccc1', 'Clc1cc(Br)cnc1-n1cccn1', 'O=[N+]([O-])c1ccc(/C=C\\c2cc(Br)ccc2O)cc1O', 'Clc1ccc(Cc2ccc(CCn3nc(-c4ccccc4O)o3)o2)cc1', 'Clc1ccc(Br)c(OCc2csc(-c3cccn3CCC(F)(F)F)n2)c1', 'COc1ncncc1-c1ccc(Br)c(S(=O)(=O)N2CCOCC2)c1', 'CNS(=O)(=O)Cc1ccc(C(=O)N2CCC[C@@H](Nc3cc(C)ccc3C)C2)cc1', 'O=C(/N=c1\\sc(C(Cl)(Cl)F)nn(CN2CCCC2)c1=O)c1cc(Br)sc1Cl', 'O=C(CS(=O)(=O)c1cnn(-c2ccccc2)c1)NCCc1c[nH]c2ccccc12', 'Nc1[nH]ncc1Sc1ccccc1', 'Clc1ccccc1N1N=C(Sc2ccccc2Cl)SC1=S', 'Clc1ccccc1-c1cscc1C(=O)Nc1ccc(C(=O)[O-])cc1', 'Clc1ccccc1N1Nc2ccccc2NC1=O', 'O=C(C1CCC(Nc2nccs2)CC1)N1CCC(n2ccccc2=O)CC1', 'O=[N+]([O-])c1cc(Cl)nc(Nc2ccc(Br)cc2F)c1', 'Clc1cccc2nsnc12', 'NS(=O)(=O)NCCOC1(c2ccccc2)CCC1', 'Clc1nnnc(-n2cnnc2COC(=O)c2cccc(OCc3ccccc3)c2)c1Cl', 'OCS(=O)(=O)c1c(Cl)cccc1Cl', 'Clc1cccc2cc(-n3nnnc3NCCN3CCCCC3=O)ccc12', 'Clc1ccccc1-n1nnnc1SCc1cc(Cl)c2c(c1)OCO2', 'O=C(/C=C/OCc1ccncc1)c1c(Cl)cccc1Cl', 'Clc1ccccc1Oc1cc(Nc2ncno2)ccc1F', 'Oc1cc(C=Nc2ccccc2)cc(I)c1O', 'CN[C@H](Cc1ccc(C)cc1)c1ccc(C)o1', 'ClC(F)(F)C1CCN(Cc2csc(-c3ccc(F)cc3)n2)CC1', 'Clc1ccc(-c2cc[nH]n2)c(/C=C/c2ccccc2)c1', 'Clc1ccc(-c2n[nH]cc2CNc2ccc(Cl)c(F)c2F)cc1', 'O=[N+]([O-])c1cc(C(F)(F)F)ccc1Oc1ccc(Oc2ccccc2)[nH]c1=O', 'O=[N+]([O-])c1ccc(Cc2nc(-c3ccco3)no2)c(F)c1', 'Clc1cccc(/C(c2ccccn2)N2CCN(CC(=O)Nc3ccc(Cl)cc3)CC2)c1', 'Oc1cc(Br)c(OCc2cc(F)cc(F)c2)c(O)c1', 'O=C(Cc1c(Cl)n[nH]c1Cl)NCc1cccc(Cl)c1', 'O=C(/N=C\\c1ccccc1[N+](=O)[O-])Nc1ccc([N+](=O)[O-])cc1', 'Clc1ccsc1Sc1cccc(F)c1', 'O=C(Cc1conc1-c1cnn(C(C)(C)C)c1)N1CCC(NC(=O)c2cccnc2)CC1', 'CNS(=O)(=O)[C@@H](C)C(=O)NCc1ccccc1', 'Clc1ccc([C@@H](OC[C@@H]2CCCC[NH2+]2)c2c[nH]c3ccccc23)cc1', 'O=C(CN(CCS)Cc1ccccn1)c1cccc([N+](=O)[O-])c1', 'Clc1ccc(Br)cc1N(Cc1ccc(Br)o1)C1CC1', 'Clc1ccccc1[C@H]1[C@@H](C(=O)c2ccccc2)O[C@H]1c1ccccc1', 'Clc1nnnc(Cl)c1Cc1ccc(Cl)cc1', 'O=C(Cc1cc(F)ccc1F)N1CCC[C@@H]1c1cccc(F)c1F', 'O=[N+]([O-])c1ccncc1Cn1cccn1', 'Oc1ccc(Oc2cn(Cl)nc2-c2c[nH]c3ccccc23)cc1', 'NS(=O)(=O)/C=C/Cc1ccccc1', 'OCc1n[nH]c(=O)c2cc(S(=O)(=O)N3CCCC3)ccc12', 'O=C(Cc1cnc[nH]1)NNC(=O)c1cccs1', 'Clc1ccc(Cc2nccs2)nc1', 'OC[C@@H](Cn1cc(Cl)cn1)NCc1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnc2c1cnn2-c1ccccc1', 'Clc1cccc([C@@H](N[C@H]2CCCc3ccc(F)cc32)C2CC2)c1', 'O=[N+]([O-])c1c(Cl)nn(-c2ccccc2)c1Nc1ccc([N+](=O)[O-])cc1', 'O=C([O-])/C(=C\\c1ccc2ccccc2c1)C1CCCCC1', 'Clc1ccc(Cc2ncc(C(F)(F)F)s2)cc1', 'Clc1ccc(-c2csc3[nH]c(=O)c(-n4cccn4)cc23)cc1', 'Clc1cc(Cl)cnc1-c1ccc(OC[C@H]2CCCO2)cc1', 'C=CCCC(=O)Nc1ccc2c(SCC(F)F)cc(C)nc2c1', 'O=C(CS(=O)(=O)Cc1nnc(-c2ccoc2)o1)Nc1ccccc1-c1ccccc1', 'Nc1nc2c(n1Cc1ccc(C(F)(F)F)cc1)CCC2', 'ClC[C@H](O)c1ccc(Cl)cc1', 'Oc1cc(COc2ncnc3c2cnn3Cc2ccccc2)on1', 'Nc1c(Oc2nc(N)nc3ccccc23)cccc1I', 'O=C(CN(CC(F)F)c1ccccc1)Nc1ccc2c(c1)C(=O)NCC2', 'Oc1ccc([N+](=O)[O-])c(-c2csc(N3CCCCC3)n2)c1', 'ClCc1cn[nH]c1C1CCN(c2cccnc2)CC1', 'Clc1ccc([C@@H](Cl)C(=O)Nc2ccc(F)c(F)c2)cc1', 'Cc1cnc2c(NCc3ccccc3)nnn2c1', 'ClCc1nonc1Nc1ccc2c(c1)CCCC2', 'O=[N+]([O-])c1ccncc1CCl', 'Clc1ccccc1Oc1cnc2nc(-c3ccc(Br)o3)[nH]c2c1', 'CNC(=O)CSc1nc(-c2ccccc2C)sc1C', 'Clc1cc(Cl)nc2[nH]nc(-c3ccccc3)c12', 'NS(=O)(=O)CCCSc1nc2cc(Cl)ccc2c(=O)[nH]1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1-c1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1-c1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1-c1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1-c1ccccc1', 'Clc1ccc(-c2n[nH]cc2C2CC2)cc1', 'O=C(CSCCO)N1CCC[C@H]1c1cccc(Cl)c1', 'CNS(=O)(=O)CCNC(=O)N(C)Cc1ncc[nH]1', 'O=C([C@H](CSC(C)=O)c1ccc(F)cc1)N1CCc2ccc(F)cc2C1', 'O=C(c1c[nH]nc1-c1ccc(Br)cc1)N1CCN(S(=O)(=O)c2ccc(F)c(F)c2)CC1', 'Clc1ccc(-c2n[nH]cc2Br)cc1', 'Clc1ccc(-c2n[nH]cc2Br)cc1', 'Clc1ccc(-c2n[nH]cc2Br)cc1', 'Clc1ccc(Br)cc1/N=C/c1ccc2c(c1)OCO2', 'Nc1c(F)cccc1N/C=C(\\C#N)C(=O)Nc1ccc(Cl)cc1', 'Clc1nnn(-c2ccc(F)cc2)c1N1CCOCC1', 'OC[C@@H](C(=O)NCCCn1cccn1)Nc1ccccc1F', 'O=C(CSCNS(=O)(=O)c1cc(F)ccc1F)Nc1ccccc1F', 'Oc1cc(-n2nnc3c(F)cccc32)c([N+](=O)[O-])c2ccccc21', 'Cc1c([C@@H](C)NC(=O)c2cc(C)n(-c3ccc(Cl)cc3)c2C)c(=O)n(-c2ccccc2)[nH]1', 'O=[N+]([O-])c1ccn(CC[C@@H](O)COc2ccccc2)n1', 'O[C@@H](Sc1ccccc1)C(=O)N(Cc1ccc(Cl)cc1)C1CC1', 'O=C(c1ccc(F)cc1F)C(=O)Nc1ccc(C(=O)[O-])cc1', 'Clc1cc(Cl)c2c(c1)CCN2C1CCC(CN2CCOCC2)CC1', 'O=[N+]([O-])c1cc(SCc2ccco2)nc(Oc2ccccc2)c1', 'O=[N+]([O-])c1c(F)cc2[nH]ccc2c1Oc1ccccc1', 'Clc1ccccc1-c1nc(CN2CCN(Cc3ccnc(F)c3)CC2)c[nH]1', 'O=C(c1c[nH]nc1-c1ccccc1)N1CCN(c2cnccn2)CC1', 'Clc1ccccc1NS(=O)(=O)c1c[nH]c2nccc(Cl)c12', 'Clc1ccc(Br)cc1Cc1nnnn1C1CC1', 'Clc1ccccc1-c1n[nH]c2cc[nH]c12', 'Clc1ccc(Br)cc1-n1ccnc1CO', 'O=C(c1ccc[nH]c1=O)N(Cc1nc2ccccc2n1Cc1ccccc1)S(=O)(=O)c1ccccc1', 'Clc1ccccc1O/C=COc1ccc([N+](=O)[O-])cc1', 'O=C(c1cnccc1Cl)N1CCSc2cc(Cl)ccc21', 'O=C(CCc1ccccc1)NCCc1nnc(-c2ccc(Cl)s2)n1-c1cccnc1', 'Clc1ccc(-c2cccnc2N2CCCSC2)cc1', 'O=C(c1cnn2cc[nH]c12)OCc1cnn(-c2ccc(Cl)cc2)c1', 'Oc1cc(F)nc(Nc2ccc(C(F)(F)F)cc2)c1', 'Clc1ccc(CCN(Cc2ccco2)C[C@@H]2C=CCCC2)cc1', 'O=[N+]([O-])c1cc(Br)cc2cc(-c3ccc(Cl)c(Cl)c3)c(=O)[nH]c12', 'O=C(CCC(=O)c1cccc(Cl)c1)N1CCC[C@@H]1c1ccc([N+](=O)[O-])cc1', 'Clc1ccccc1Oc1cc(-c2nc3c([nH]c2=O)CCCC3)ccc1O', 'N#C[C@@]1(NC(=O)CN2CCCC2)CC1', 'ClC(=S)Nc1ccc2c(c1)OCCO2', 'Clc1ccccc1C1=CO[C@H]2C[C@@H](CC(=O)OCc3cccc(Cl)c3)N12', 'O=C(/C([O-])=N/S(=O)(=O)c1ccc(Br)cc1)Nc1ccccc1', 'O=C(/C=C/O)N1CCN(S(=O)(=O)c2c(Cl)cccc2Cl)CC1', 'Cc1nonc1NC(=O)N1CCN(c2cc(C)ccn2)CC1', 'Clc1ccc(Br)cc1-n1cnnc1', 'Clc1ccccc1Oc1cncc(N2CCC[C@H](Oc3ccccc3F)C2)n1', 'Oc1ccc([C@H](OC(=O)Cc2ccccc2)N2CCOCC2)c2c1CCCC2', 'O=[N+]([O-])c1c(-c2ccco2)sc2ccccc12', 'O=C(c1cnn(C[C@H]2CCCO2)c1)N1CCCCC1', 'ClCc1csc2nc(-c3ccccc3)nn12', 'Clc1ccc(Br)cc1OS(=O)(=O)c1cccnc1Cl', 'NS(=O)(=O)N(Cc1ccc(Cl)cc1)C1CCSCC1', 'Oc1ccccc1[C@H]1CN(c2ccccc2)CCN1CCn1cc[nH+]c1', 'Oc1ccccc1/N=c1\\scc(N2CCOCC2)n1-c1ccccc1', 'O=C([C@H]1CCCN1C(=O)c1cnc2ccccc2c1)N1CCN(c2ccccc2C(F)(F)F)CC1', 'Clc1cccc2[nH]nc(Sc3nc4ccccc4s3)c12', 'O=C(c1cccc(C2CCOCC2)c1)N1CCC(/C(Cl)c2ccccc2)CC1', 'COC[C@@H](C)NC(=O)c1nn(-c2ccccc2)nc1C', 'CCCC[C@H](CC)c1ccc([N+](=O)[O-])cc1', 'O=C(c1cccnc1)N1CCC[C@H](c2nccn2Cc2ccccc2)C1', 'Clc1ccc(Br)c(Cl)c1OCCN1CCN(c2cccc[nH+]2)CC1', 'Clc1cc(Nc2ncc[nH]2)nc2ccccc12', 'O=C([C@H](CCc1cccnc1)NCc1ccccc1)N1CCCC[C@H]1c1ccccc1', 'COc1ncnc(Br)c1C(=O)N1CCC(OC)CC1', 'O=[N+]([O-])c1c(Br)cnc2ccc(F)cc12', 'Clc1cccc(-c2cnsc2)c1', 'Clc1ccc(Br)c(F)c1-c1cc([N+](=O)[O-])cnc1-c1ccccc1', 'Oc1ccc([N+](=O)[O-])c(/C=C/Nc2ccccc2Br)c1', 'Clc1ccccc1-c1cscc1-c1cc2c(cc1Cl)OCCO2', 'O=C(/N=C\\c1c(F)cccc1Cl)c1cccc2cccnc12', 'Clc1ccc(C(=O)N2CCCNC2)cc1F', 'O=C([C@H](Sc1ccccc1)c1ccc(F)c(Cl)c1)NC1CCOCC1', 'Oc1cc(F)ncc1-c1cccnc1NCc1ccccc1F', 'O=C(Cc1nnnn1-c1ccccc1)N1CCO[C@H](c2ccccc2F)C1', 'Clc1ccccc1NCc1ccc([S@](=O)C2CCC2)cc1', 'O=C([O-])[C@@H](Cc1ccccc1)N1CCc2ccccc21', 'O=C(CCOn1ccccc1=O)Nc1cc2c(cc1Cl)OCCO2', 'Clc1ccsc1SCc1ccc(Cl)cc1Cl', 'O=[N+]([O-])c1cc(N[C@@H]2CCN(C(=O)c3ccccc3F)C2)ccc1F', 'Clc1nnnc(-c2ccc(Cl)cc2)c1-c1cccc(C(F)(F)F)c1', 'Oc1ccccc1/N=N\\c1ccccc1Br', 'ClCc1csc(Cc2cccs2)n1', 'O=C(CNc1nnccn1)Nc1c(F)cccc1F', 'Clc1ccccc1OC1CCC([NH2+]Cc2ccccc2CO)CC1', 'Clc1ccc([C@@H]([NH2+]Cc2ccccc2)C2CCN(C3CCCCC3)CC2)cc1', 'O=C(/N=C/CCN1C(=O)c2ccccc2C1=O)NCc1cccc(Cl)c1', 'Clc1ccc(-c2cc(SCc3nc4ccc(F)cc4o3)n[nH]2)cc1', 'ClCc1nnsc1C(=O)NC1CCCCC1', 'Clc1ccccc1[C@H](OCc1ccco1)C1CC1', 'Cc1ncsc1CCN(C)C(=O)CCNC(=O)OC(C)(C)C', 'Clc1cc(Cl)cnc1SCC1CCOCC1', 'Cc1snnc1CN1CCC(C(=O)NC2CCCCC2)CC1', 'CN(C)C(=O)C(=O)Nc1ccc(-c2nc3ccccc3s2)c(C(F)(F)F)c1', 'N=Cc1nc(-c2ccccc2)cs1', 'O=C(Cc1n[nH]nc1C1CC1)NCC1CC[NH+](CC2(c3ccccc3F)CCCC2)CC1', 'Clc1cc(Br)c2nc(OC3CCCC3)[nH]c2c1', 'Clc1ccc(C(=O)N2CCN(Cc3c[nH]nc3C3CC3)CC2)c(F)c1', 'Nc1cn[nH]c1Sc1ccccc1', 'OCSc1nc2ccccc2cc1NCc1cccc(Br)c1', 'Clc1nncn1c1cc(Cl)ccc1Cl', 'Clc1ccccc1C(=O)O[C@H]1CCOc2ccccc21', 'COCCS[C@H](C)C(=O)N[C@H]1CCc2ccc(C)cc21', 'O=[N+]([O-])c1cc(Cl)c(OCC(F)(F)F)cc1Br', 'Clc1ccc([C@@H](Cl)Cc2cccs2)cc1', 'Clc1ccccc1O/C=C(O[C@H]1CCSCC1)c1ccccc1', 'Clc1ccc(C(F)F)cc1NC[C@H]1CCCN(c2ccccc2)C1', 'Clc1ccc(-c2n[nH]cc2CSc2n[nH]c(=O)n2CCc2ccccc2)cc1', 'Clc1cc(Cl)nc(CSc2cc(F)ccc2Cl)c1', 'COc1ncc(CN2CCN(C(=O)c3cnn(C(C)(C)C)c3)CC2)cc1Br', 'Oc1ccc(F)cc1-n1nc(C(C)(C)COC2CCCCC2)c2ccccc21', 'Clc1ccccc1-c1cscc1-n1ccnc1', 'Clc1ccc(-c2cc[nH]n2)c(CCC(=O)N2[C@H]3CCn4c(nc5ccccc54)[C@@H]3C2Cc2ccccc2)n1', 'O=C(c1cnn(Br)c1)c1ccc(Cl)cc1', 'O=C(c1cc([N+](=O)[O-])ccc1Cl)N1CCN(Cc2cc(C3CC3)cs2)CC1', 'Clc1ccccc1Oc1cc(Br)c[nH]1', 'Clc1ccc(-c2nonc2CNc2cc(Cl)ccc2Cl)cn1', 'O=C(c1cc(Br)cnc1Cl)N1CCC(c2ccccc2)=N1', 'Clc1cccc2cc(O)[nH]c12', 'Clc1ccc(Br)cc1/C=N\\OCCN1CCCC1=O', 'ClCc1csc(OCc2ccc(Cl)cc2Cl)n1', 'NS(=O)(=O)CCN(C(=O)Cc1ccco1)Cc1ccccc1Br', 'Clc1ccc(COc2nc(F)cc(Br)c2)c(F)c1', 'Clc1ccc(-c2csc(NC[C@@H]3CCCCO3)n2)cc1', 'Clc1ccsc1CNc1ccc(F)c(C(F)(F)F)c1', 'O=C(CSCCN1C(=O)COc2ccccc21)Nc1nccs1', 'O=C(c1cc(Cl)c2[nH]ccc2c1)N1CCN(c2cnccn2)CC1', 'O=C(CCC(=O)N1CC[C@@H](CCCO)C1)Nc1cccc(OCC(F)(F)F)c1', 'O=[N+]([O-])c1c(Br)cccc1C(F)(F)F', 'Oc1cc(F)cc2c1cnn2Cc1cccs1', 'Nc1c(CSc2nnc(-c3ccccc3)cn2)oc2c1CN(Cc1ccc(F)cc1)CC2', 'O=[N+]([O-])c1c(-c2ccsc2)ncnc1NCc1ccccn1', 'Oc1ccc([N+](=O)[O-])c(S(=O)(=O)NCc2c(O)cccc2O)c1', 'Clc1cc(Cl)c2sc(-c3ccc(CCN4CCOCC4)o3)nc2c1', 'Clc1nnc(/C=C/c2ccccc2)n1/N=C/c1ccco1', 'CNS(=O)(=O)CC(=O)N[C@H](C)c1ccc(OC)c(F)c1', 'Clc1ccccc1NCc1c[nH]nc1Cc1cccc(Br)c1', 'ClCc1ccnnc1Cn1ccc2ccccc21', 'Oc1cc(OCCn2cccc2Cl)c2ccccc2n1', 'Clc1ccccc1NC[C@@]1(O)Cc2ccccc2C(=O)N1c1ccccc1', 'O=[N+]([O-])c1ccc(N[C@H](c2ccccc2)C2CC2)cc1F', 'O=C(CS(=O)(=O)c1c[nH]c2ccccc12)NNC(=O)/C=C/c1ccco1', 'Oc1cccc(Br)c1C(F)(F)F', 'Oc1cccc(Br)c1C(F)(F)F', 'COCc1cccc(NC(=O)C(=O)N2CCC[C@H](C(=O)Nc3ccccc3)C2)c1', 'O=[N+]([O-])c1c(Br)cc(F)cc1C(=O)Nc1cc(C(F)(F)F)ccc1Cl', 'Clc1ccccc1Cc1ncsc1Br', 'Oc1ccc(F)c(Oc2cc(F)ccc2F)n1', 'Clc1ccc([C@@H](Br)c2cc(F)ccc2F)cc1', 'O=C(C[NH+]1CC[C@@H]2CCCC[C@@H]21)NC[C@H]1-c2ccccc2CN1C(=O)c1ccc(F)cc1', 'Clc1cc(Cl)ncc1Br', 'Clc1ccccc1-c1noc(CSc2nnc(C3CC3)s2)n1', 'Clc1cccc(Nc2cnccc2S(=O)(=O)/N=C/c2ccc(F)cc2)c1', 'O=C(CNCc1c(O)cc([N+](=O)[O-])cc1F)Nc1cccc(O)c1', 'Clc1ccccc1Nc1cc(S(=O)(=O)N2CCCC2)cc(S(=O)(=O)[O-])c1', 'ClCc1cc(OCc2cccc(Cl)c2)nn1CC1CC1', 'O=C(c1cccs1)C1CCN(c2nc3ccccc3oc2=O)CC1', 'Oc1ccc(/N=C/c2cccc(Cl)c2C)cc1Cl', 'O=C(CCOn1cccn1)Nc1ccc2c(c1)OCO2', 'Clc1cc(/C=N\\c2ccc(F)cc2F)ccc1[N+](=O)[O-]', 'ClC[C@H]1CCCN(Cc2csc(-c3ccccc3)n2)C1', 'Clc1nnnn1c1cccc(C(F)(F)F)c1', 'Clc1ccc(-c2n[nH]cc2Cn2cnc3ccc(F)cc32)cc1', 'OC[C@@H](C(=O)N1CCc2ccccc2C1)N1CCc2ccccc2C1', 'Clc1ccc([C@@H](C[C@H]2CCCO2)Sc2ccccc2)cc1', 'O=C(/N=c1/oc(Cc2ccccc2)cc2c1N[C@H](c1ccco1)S2)c1ccc(F)cc1', 'Clc1ccc(C[NH2+]CCNc2ccc(Br)cc2)cc1', 'Clc1nnnn1c1c(Br)cccc1N1CCCCC1', 'O=C(CN(c1cnc(C(=O)NCc2ccccc2)c(Cl)c1)S(=O)(=O)c1ccoc1)c1ccccc1', 'Clc1ccc(Br)cc1-c1cn(-c2ccccn2)nc1C1CC1', 'Nc1nnc(Cl)n1C1CCCCC1', 'Oc1cccc(NCC(=O)N2CCC[C@@H]2c2ccc(Cl)c(Cl)c2)c1', 'Clc1ccccc1OCCNC(=O)[C@@H](c1ccco1)C(F)(F)F', 'Clc1ccccc1Nc1ncnc2[nH]c(C3CC3)nc12', 'Oc1ccc(F)c(F)c1Br', 'ClCc1cc(Oc2c(Cl)cccc2Cl)ncn1', 'Clc1ccccc1-c1cscc1C(=O)N1CCc2c([nH]c3ccccc23)C1', 'O=[N+]([O-])c1ccc(CN2CCN(c3ncnc4scnc34)CC2)cc1', 'Oc1ccc([N+](=O)[O-])c(Cl)c1Cl', 'O=[N+]([O-])c1c([N+](=O)[O-])cc([N+](=O)[O-])c2ccccc12', 'O=C(COCc1ccccc1)NC[C@H]1CCC[C@@H](O)C1', 'Clc1cccc2cc(CNc3cccnc3Cl)sc12', 'ClCc1cn(C2CCN(c3ncccn3)CC2)cn1', 'Oc1ccc(Oc2ncn[nH]2)cc1', 'Oc1ccc(F)cc1[C@H]1CCCN1c1ccccc1', 'Oc1cc(COc2ncnc(-c3cccs3)n2)ccc1O', 'O=C([O-])c1[nH]ncc1Cn1c(CO)nc2ccc(-c3ccccc3)nc21', 'O=[N+]([O-])c1ccncc1C(F)(F)F', 'O=[N+]([O-])c1cc(N2CCC(Oc3cc(F)cc(Cl)c3)CC2)ccc1OC[C@@H]1CCCO1', 'NS(=O)(=O)Cc1cc(Cl)ccc1-c1nc(-c2ccncc2)cs1', 'ClCc1cn(CCO)nn1', 'ClCc1csc(Oc2cccc(F)c2)n1', 'O=C(CNS(=O)(=O)c1ccccc1)Nc1cccc(-c2nc[nH]n2)c1', 'NS(=O)(=O)Nc1cc(Cl)ccc1C(=O)NC[C@@H]1CCCO1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1OC1CC1', 'NS(=O)(=O)Nc1cc(Br)ccc1C(F)(F)F', 'Cc1c(Br)oc(C)c1C(=O)NC(=S)NC(=O)COc1ccccc1', 'Clc1ccc(-c2nonc2NC[C@H]2CCCO2)cc1', 'Oc1ccc(F)c(Cl)c1Cn1ccnc1', 'Clc1ccc([C@@H](Br)CCCO)cc1', 'N[C@@H](COCc1ccc(F)cc1)c1cc(Cl)ccc1Cl', 'Clc1ccc(Br)cc1/C=N\\N=C\\C=C\\c1ccc(Br)cc1', 'Oc1ccc(/C=N\\Cc2ccccc2)c(Br)c1O', 'N=Cc1cn[nH]c1SCC(=O)NCc1ccccc1', 'Oc1cccc(-c2cnn(-c3ccc([N+](=O)[O-])c(NC4CCCCC4)c3)n2)c1', 'O=[N+]([O-])c1c(Cl)cccc1C(=O)Nc1ncc[nH]1', 'Oc1ccccc1-n1cc(/N=C/c2cccs2)c2ccccc21', 'Oc1ccccc1OC[C@H](O)COc1ccc2ccc(=O)oc2c1', 'Oc1ccc(F)c(Oc2ccccc2C(=O)[O-])c1', 'Clc1nnnc(COc2ccccc2Cn2cccn2)n1', 'Clc1ccccc1Cc1ncoc1-c1cc(F)cc(F)c1', 'Clc1ccc(Br)cc1-n1nnc(-c2cccs2)n1', 'ClC([O-])=C1CCC(n2c(=O)[nH]c3ccccc32)CC1', 'O=C(COc1cccc(C(F)(F)F)c1)N1CCC[C@H]1c1ccccn1', 'Clc1ccc(-c2csc3[nH]ncc23)cc1', 'Clc1ccccc1[C@H](Cl)c1ccc(Cl)cc1F', 'Clc1ccccc1-c1cscc1Cn1ccnc1C1CC1', 'ClCc1csc(C2CC2)n1', 'ClCc1csc(C2CC2)n1', 'O=C(c1c[nH]cn1)N(CCNc1ccc(OC(F)(F)F)cc1)C1CC1', 'Oc1cc(O[C@@H]2OC(=O)c3ccccc3C2)ccc1OCc1ccccc1', 'Clc1cccc2[nH]nc(S/c3ccc(F)c(F)c3)c12', 'Oc1cc(C=NC(=O)c2cccc(Br)c2)ccc1OC(F)F', 'O=C(c1c[nH]nc1-c1ccccc1F)N1CCO[C@@H](c2ccccc2)C1', 'O=C([C@H](Cl)Br)c1ccccc1', 'Clc1ccc(Br)cc1OC[C@H]1CCCO1', 'Oc1cc(F)cc(NC2(c3cccc(F)c3)CCC2)c1', 'NS(=O)(=O)/N=C/C=C/c1ccc(Cl)cc1OCc1ccccc1', 'O=[N+]([O-])c1c(F)ccc(CNc2ncccc2F)c1Cl', 'Clc1ccc(Br)c(SCCn2cncn2)c1', 'O=C(/N=C/c1ccc(-n2cccn2)cc1)N1CCC(O)CC1', 'Clc1cc(Br)c(N/C=C/c2ccccc2)c2[nH]ccc12', 'Clc1cccc([S@@]CC=N/c2ccc(Cl)cc2Cl)c1', 'Oc1ccc(O[C@@H](Cc2ccc(Cl)cc2)c2cccc(Cl)c2)cc1F', 'Clc1ccc(Br)cc1-n1ncc2sccc21', 'Oc1cccc(Br)c1S(=O)(=O)NCc1cccs1', 'Clc1ccc(-c2cccnc2)n2nnc(-c3ccc([N+](=O)[O-])cc3)c12', 'OCc1n[nH]c(NCc2ccccc2Cl)n1', 'Clc1ccccc1Oc1cc(Br)c(N2CC[NH+](CC(=O)NC(C)(C)C)CC2)cc1', 'Clc1ccc(Br)cc1N1C(=O)/C(=C\\c2cccnc2)SC1=S', 'Clc1ccccc1-c1cncc(CN2CCC[C@H]2c2ccccc2F)n1', 'Oc1ccc(F)cc1Sc1nc2ccc(C(F)(F)F)cc2s1', 'CN(Cc1cccc(Cl)c1)C(=O)c1c[nH]nc1-c1ccc(OC(F)F)cc1', 'Clc1ccccc1OCCSc1nc(-c2ccco2)n[nH]1', 'O=C([O-])/C(=N/Nc1ccccc1Cl)c1ccncc1', 'Clc1nncn1c1cnn(C2CCC(Oc3ccc(Cl)cc3)CC2)c1', 'Clc1ccccc1CO[C@H]1CCC[C@H]1Cc1ccc(C(F)(F)F)cc1', 'Clc1ccsc1-c1cn2ccccc2n1', 'COc1nn[nH]c1CCCNC(=O)c1ccnc(Br)c1', 'CN(Cc1ccc([N+](=O)[O-])cc1)C(=O)NCc1ncn(C)n1', 'Clc1ccccc1CNc1nnc(Cn2ncnc2-c2cccnc2)s1', 'O=[N+]([O-])c1c(-c2nc3ccccc3o2)ncnc1N1CCOCC1', 'Clc1ccccc1[C@H](C[NH+](Cc1ccccc1)C1CCCC1)Nc1cc(F)ccc1Br', 'Clc1ccccc1OCc1csc2[nH]nc(-c3cccnc3)c12', 'Clc1ccc(C(C)(C)CN2CCNC[C@H]2c2ccccc2)cc1', 'Oc1ccc([N+](=O)c2ccc(Cl)cc2Cl)cc1', 'Clc1ccccc1Oc1cnc(I)cc1O', 'Oc1cc(-n2nncc2CN2CCO[C@@H](c3ccccc3)C2)ccc1N1CCCC1', 'Clc1cccc(CN2CCCC[C@H]2c2nc3ccccc3o2)c1', 'Oc1ccc(/N=c2/[nH]cccc2CCNC(=O)c2cccnc2)cc1', 'O=C(Cc1n[nH]c(=O)[nH]1)NC1(c2ccccc2)CC1', 'Clc1ccsc1S(=O)(=O)N1CCC(N(Cc2ccncc2)C(=O)c2ccc(F)cc2)CC1', 'Clc1ccc(-c2cc([N+](=O)[O-])ncc2N2CCCC2)cc1Cl', 'O=[N+]([O-])c1c(F)cc(N/N=C/c2sc3ccccc3c2CO)cc1OCc1ccccc1', 'O[C@@H](c1cc(Br)ccc1F)c1cccc(F)c1', 'Oc1ccc(F)cc1CC1CCN(C(Cc2ccccc2)Cc2ccco2)CC1', 'Clc1ccc(Br)cc1-c1cnn2cc(Br)cnc21', 'CN(CCCNC(=O)[C@@H]1COc2ccccc21)C1CC1', 'Clc1ccccc1[C@H](NNC(=O)C1CCC1)c1ccccc1', 'O=C(/N=C\\c1occc1OCc1ccc(Cl)cc1)C1CCCC1', 'O=[N+]([O-])c1cc(Cl)ccc1-c1nc(C(F)(F)F)cs1', 'COC[C@@H](C)NC(=O)c1ccnc(OCC(C)C)c1', 'Clc1ccccc1Oc1cc([N+](=O)[O-])cc2c1OCOC2', 'Clc1cc(N2CCc3sccc3C2)ccc1C(F)(F)F', 'Oc1cccc(N2CCN(Cc3ccc(Cl)nc3Cl)CC2)n1', 'O=C(c1ccc[nH]c1=O)N1CCN(c2nccc(Br)n2)CC1', 'Oc1cc(-n2ncc(Br)c2Cl)ccc1-c1ccccc1O', 'Clc1ccc(-c2nonc2Cn2cccc(Br)c2=O)cc1', 'O=[N+]([O-])c1c(Cl)nc(Cl)cc1Oc1cccc(Cl)c1Cc1ccccc1', 'ClCc1nc(Oc2ccc(Cl)cc2)nnc1-c1cccc(Cl)c1', 'O=C(c1c[nH]c2nc[nH]c12)c1ccc(Br)cc1', 'CN[C@H](c1cccc(OC)c1)c1cc(F)cc(F)c1', 'Clc1cc(COC[C@@H]2CCCO2)ccc1NCc1coc(-c2ccccc2)n1', 'Clc1nnn(-c2ccc(F)cc2)c1I', 'Clc1ccccc1-c1nccn1C1CCN(S(=O)(=O)c2cccc(Cl)c2F)CC1', 'O=C([C@H](Nc1nc(-c2ccccc2)n[nH]c1=O)Nc1ccccc1)N1CCCCC1', 'Oc1ccc(F)c(F)c1CNC[C@@H](O)c1ccc(Cl)cc1', 'Clc1ccc(Br)c(N2CSCC2)c1', 'O=C(CNc1nn(C2CC2)c(=O)c2ccccc12)NCc1csc(Nc2ncccn2)n1', 'Clc1ccccc1Cc1nc2ccccc2nc1NC[C@H]1COCCO1', 'Oc1ccc(/C=C/N=C(\\[O-])COc2ccccc2)cc1', 'O=C(/N=C\\Nc1cccc(N2CCCC2)c1)c1ccc2c(n1)CCN(Cc1csc(Cl)c1)C2', 'Clc1ccccc1Nc1cnc(Br)s1', 'O=[N+]([O-])c1cc(Cl)n(-c2ccc(F)cc2)n1', 'OCc1n[nH]c(O)c1-c1ccccc1', 'Clc1ccc(Br)cc1/N=C/c1cc(C(F)(F)F)ccc1SC1CC1', 'O=[N+]([O-])c1c(Cl)nn(C[C@H]2CCCO2)c1N1CCN(Cc2ccccc2)CC1', 'Oc1cccc(Br)c1OCc1c[nH]c2nccc(Oc3ccccc3C(F)(F)F)c12', 'C[C@@H](NS(=O)(=O)c1ccc(C(=O)N2CCCC2)cc1)c1cccc(C#N)c1', 'C[C@@H](Oc1ccc(Cl)cc1Cl)C(=O)Nc1ccc(C(=O)O[C@H](C)C(F)(F)F)cc1', 'O=[N+]([O-])c1ccc(/C=C/CCCSc2ccccn2)c([N+](=O)[O-])c1', 'Clc1cc(CNC(=O)c2cc(C3CCCCC3)no2)cc([N+](=O)[O-])c1O', 'O=C(CS(=O)(=O)CCC1CCCCC1)NNc1ccc2c(c1)COC2', 'Clc1ccc(OC[C@H]2CO2)cc1F', 'Clc1ccc(-c2nn(-c3cc(Cl)ccc3Br)nc2N2CCOCC2)cc1', 'Clc1ccccc1O/C=CCN1C(=O)CN(c2ccc(Cl)cc2)C1=O', 'O=C(CS(=O)(=O)Cc1cccc(Cl)c1)Nc1ccccc1C(=O)[O-]', 'Oc1cccc(SCCc2nnc(-c3ccccc3[N+](=O)[O-])o2)c1', 'N=C1\\C(=C\\c2ccc(C#CC[NH+]=C/c3ccccc3)cc2)NC[C@@]1(O)Cc1ccccc1', 'O=C(c1ccsc1I)N(CC1CC1)C1CC1', 'Nc1c(Oc2ccc(C(N)=O)nc2CO)nc2ccccc2c1Cl', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1-c1ccc(F)cc1', 'C[C@@H]([C@@H](O)c1ccccc1)N(C)CC(=O)Nc1ccc(F)cc1Cl', 'O=[N+]([O-])c1c(-c2cccs2)nnn1-c1ccccc1', 'Clc1cccc2nc(-c3cc(F)c(F)cc3Br)[nH]c12', 'Clc1cccc(F)c1N1CCOc2cccc(F)c21', 'Clc1ccccc1N1N(CC2CC2)C(=O)N1Cc1cccc(F)c1', 'Clc1ccccc1N1Nc2ccccc2C(=O)N1Cc1cccc(F)c1', 'Clc1cc(Br)c2nc(OCc3ccccc3F)[nH]c2c1', 'Clc1ccc(C(F)(F)F)n1NCC1CCN(C(=O)CC2=COCCO2)CC1', 'Clc1ccccc1CCCn1c(C[C@@H]2CCS(=O)(=O)C2)nc2ccccc21', 'Oc1cccc(-c2cncc(Cl)n2)c1O', 'Clc1ccc(Br)cc1-c1cnc2n1CCN(Cc1cccc([N+](=O)[O-])c1)C2', 'Cc1nonc1C(=O)Nc1cc(C(=O)NCCC#N)ccc1Br', 'CN=C/S(=O)(=O)Cc1ccc(OC(C)C)cc1', 'Clc1ccc(Cc2csc(-c3c[nH]nc3N3CCCC3)n2)cc1', 'O=[N+]([O-])c1c(Br)cc(Cl)cc1C(=O)N1CCOCC1', 'Clc1ccc(CCOC(F)F)c(Br)c1', 'Clc1cc(Br)c2nc(Oc3ccccc3)cnc2n1', 'ClCc1nnc(-c2cccs2)o1', 'Cc1nonc1NC(=O)/C=C/c1cc(C)cc([N+](=O)[O-])c1', 'Clc1ccccc1O/N=C/c1cc(Br)ccc1OCn1cccn1', 'COc1nnnn1-c1cncc(NCc2ccc(C(F)(F)F)cc2Cl)c1', 'O=C(COc1cc(Br)ccc1OCc1ccccc1)N(Cc1ccc(F)cc1)S(=O)(=O)c1ccco1', 'Clc1ccc(-c2cc(-n3ccnc3)cn3cnnc23)cc1', 'Clc1cc(Br)c(N2CCN(Cc3cscn3)CC2)cc1F', 'O=C(c1c[nH]c2nccc(Cl)c12)N1CCC(c2nc3ccccc3[nH]2)CC1', 'Clc1ccccc1-c1noc([C@@H]2CN(Cn3cccc3F)C2)n1', 'Clc1ccccc1Oc1cnc2cc([N+](=O)[O-])cnc2n1', 'Clc1cc(/C=N\\c2ccc(Cl)cc2)c(F)cc1[N+](=O)[O-]', 'Clc1nnnn1-c1cc(Cl)ccc1Cl', 'O=[N+]([O-])c1c(F)cc([N+](=O)[O-])cc1OCCCCO', 'Clc1ccccc1-c1cscc1Cn1cncn1', 'Clc1ccsc1Nc1nc2ccc(F)cc2s1', 'Clc1ccc(Br)c(S(=O)(=O)N[C@@H]2CCCc3ccccc32)c1', 'Oc1cc(O)c([N+](=O)[O-])cc1OCCCC(F)(F)F', 'Clc1ccc(-c2cc([C@H](NC3CC3)C(=O)N3CCCC3)ccc2Cl)cc1', 'Clc1ccc(C(CC)(NCC(F)(F)F)C(F)(F)F)cc1', 'Oc1ccc(F)cc1SC1CCN(C(=O)c2ccc(Cl)nc2)CC1', 'Clc1ccc(Br)cc1-n1nnc2sc(-c3ccncc3)nc21', 'Clc1cccc(F)c1-n1cnnc1', 'O=C(COCc1nc[nH]n1)NCCc1ccccn1', 'CNS(=O)(=O)[C@@H]1CCN(C(=O)c2ccccc2)C1', 'Clc1ccc(Br)cc1-c1cnncn1', 'Clc1cccc2c1C[C@@H](Cc1cccnc1)C2', 'Clc1ccc(Br)c(SC[C@@H]2OCCCO2)c1', 'Clc1ccc(C(N)=N/S(=O)(=O)CCCC(=O)OC(F)(F)F)cc1', 'Clc1cccc2nc(NC[C@@H]3CCCCO3)ncc1-2', 'OC[C@@H](CO)N1CCC(c2ncc[nH]2)CC1', 'O=[N+]([O-])c1c(Cl)ncc2cc(S(=O)(=O)Nc3ccc(Br)cc3F)ccc12', 'Clc1cc(/N=C/c2ccc(I)cc2)nc(Oc2ccc(O)cc2)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1Cc1ccccc1Br', 'Oc1ccc(/C=C/NC(=S)Nc2c(F)cccc2Cl)c2ccccc12', 'O=C(c1c[nH]nc1N1CCOCC1)N1CCN(Cc2cccc(C(F)(F)F)c2)CC1', 'Clc1ccc(CN[C@@H](Cn2cnc3ccccc32)N2CCCC2)cc1', 'ClCc1cccc(N[C@@H]2CCN(c3ccccc3)C2=O)c1', 'O=C([C@H](c1ccccc1)N(CC(F)(F)F)C1CCCCC1)C1CCCC1', 'Clc1ccsc1C(=O)N[C@@H](c1ccccc1)c1ccccc1O', 'Nc1nc(CNCc2ccc[nH]2)nc2ccc(F)cc12', 'Clc1ccc(Br)c([C@@H](CO)Nc2ccc(Br)cc2)c1', 'Nc1nonc1Sc1ccccc1', 'Clc1ccc(-c2nn(-c3ccccc3)[nH]c2=O)cc1F', 'Oc1ccc([N+](=O)[O-])c(N2CCOCC2)c1F', 'Clc1cccc2c(N)[nH]nc12', 'Clc1cccc(Br)c1COC1CC[NH+](CC2=c3ccccc3=[NH+][C@@H]2c2ccccc2)CC1', 'Clc1ccccc1Oc1cc(-c2n[nH]c(=O)[nH]2)ccc1O', 'O=C([C@H](Cl)c1ccccc1)c1cc(Br)ccc1F', 'Clc1ccccc1Oc1cnc2snnc2c1', 'Clc1cc(/N=C\\c2csc(-c3ccc(F)cc3)n2)ccc1Br', 'Clc1ccc(C[NH+](Cc2ccccc2)Cc2ccccc2F)cc1', 'Oc1ccc([N+](=O)c2ccc(F)cc2F)cc1', 'Clc1cc(F)cc(Cl)c1C(F)(F)F', 'Oc1cc(-n2nncc2NCCc2ccc(CCl)cc2)ccc1O', 'CN[C@H](NS(=O)(=O)c1ccc(C)c(F)c1)c1ccc(F)cc1', 'O=C(COc1ccc([N+](=O)[O-])cc1)N[C@@H]1CCCN(c2ccc(F)cc2)C1', 'Nc1c(C2CC2)nc2scc(-c3ccccc3)c2c1C', 'ClCc1csc(Cn2ccnc2-c2ccccc2F)c1', 'NS(=O)(=O)Cc1ccsc1', 'Clc1cc(/N=C\\c2ccco2)c2ccccc2c1O', 'O=C(c1cn[nH]n1)N1CCC(NC(=O)c2ccc(Cl)o2)CC1', 'Clc1ccc(-c2csc3[nH]cnc32)c(C(=O)Nc2ccc(Br)cc2)c1', 'Clc1ccc(CO)n1CCSc1ccc(F)cc1Cl', 'Clc1cc(/N=C/Cc2ccccc2Cl)ccc1O', 'O=C(c1cnn(C[C@@H]2CCCO2)c1)N1CCN(c2ccc3c(c2)OCO3)CC1', 'Clc1ccc(C(C)C2CCN(C(=O)[C@H]3COc4ccccc4O3)CC2)cc1', 'Clc1ccccc1OC1(O)CCN(c2cccc[nH+]2)CC1', 'Oc1cc(F)ncc1SCC(=O)Nc1ccc2c(c1)OCCO2', 'O=[N+]([O-])c1c([N+](=O)[O-])c(Cl)ccc1NC1CC1', 'O=C(c1cnncc1Cl)N1CC=C(c2ccc(F)cc2)CC1', 'Clc1ccc(Br)cc1-c1cn(-c2ccccc2F)[nH]c1=O', 'Clc1cc(NSc2ncnc3ccccc23)c2ccccc2n1', 'Clc1cc(Br)c(C(F)(F)F)cc1OC(=O)C1CC1', 'Clc1ccc(CNC[C@@H]2CCCN2C(=O)c2ccc(Cl)cn2)cc1Cl', 'O=C(CNS(=O)(=O)C[C@@H]1CCCO1)Nc1ccccc1F', 'Oc1cc(F)ccc1CN1CCN(S(=O)(=O)c2cccc(OCCn3cncn3)c2)CC1', 'Oc1ccc(F)cc1N[C@H](CO)c1ccc(F)cc1', 'Oc1cccc(-c2cn(-c3ccc(Cl)cc3Cl)[nH]c2=O)c1', 'Clc1ccccc1OCCC[NH2+]Cc1ccc(OC(F)F)cc1', 'Clc1ccc(Br)cc1N(CN1CCOCC1)C1CC1', 'Oc1cccc(C[C@@H]2CCCN2C(=O)c2ccnc(NC3CCCCC3)c2)c1', 'Clc1ccc(CN2C[C@H](Cc3ccccc3)O[C@H](c3ccccc3)C2)cc1Cl', 'CCn1nnnc1[C@@H]1CCCCN1C(=O)CN(C)C(=O)c1cccnc1', 'Nc1[nH]nc(-c2cccc(Cl)c2)c1N', 'Clc1cccc(Br)c1S', 'Oc1cccc(NC2CCN(C(=O)N[C@@H]3CCN(C(=O)c4ccncc4)CC3)C2)c1', 'Clc1ccc(Cc2ncco2)cc1O', 'Clc1cc(CSc2nncn2-c2ccc(Br)cc2)c2ccc(F)cc2c1O', 'NS(=O)(=O)/C=C/c1ccc2c(c1)OCO2', 'Clc1ccccc1OCc1cc(=O)n2nc(Br)sc2n1', 'Clc1ccc(-c2n[nH]c3c2C(=O)C[C@@H](c2ccccc2)C3)cc1', 'Clc1ccc(-c2n[nH]cc2C(F)(F)F)cc1', 'COc1nn[nH]c1C(=O)Nc1ccccc1S(=O)(=O)c1ccc(F)cc1', 'Clc1ccccc1N1N(CCSc2ccc([N+](=O)[O-])cc2)CC1', 'Nc1c(F)c(F)c(F)c(F)c1F', 'O=C(/N=C/c1ccco1)c1ccnc(N2CCCCC2)c1', 'O=C(/C(CO)(CC(=O)N1CCOCC1)c1ccoc1)c1ccccc1', 'Oc1ccc(/C=N/O)cc1Br', 'Clc1cc(CCC(=O)N(Cc2cccs2)C[C@H](O)c2ccccc2)ccc1F', 'Clc1nnnn1-c1cc(S(=O)(=O)N2CCCC2)ccc1Cl', 'Clc1nnnn1-c1cc(S(=O)(=O)N2CCCC2)ccc1Cl', 'Clc1ccc(Br)cc1N1/N=C/c2ccccc2N1CCCc1ccccc1', 'Oc1cc(OC[C@@H]2CCCO2)ccc1NCc1ccc(F)cc1', 'ClC(=S)[C@@H](c1ccccc1)N1CCCc2ccccc21', 'Clc1nncc(-c2c(F)cccc2F)n1', 'Clc1ccsc1/N=C/c1cccc(OC2CCCC2)c1', 'O=C(/N=C\\NCc1nc2ccccc2s1)c1cccc2ccccc12', 'O=C(COCc1nc2ccccc2c(=O)[nH]1)Nc1nc(C(F)(F)F)n[nH]1', 'O=C(CN(C[C@H]1CCCO1)C1CC1)Nc1cccc(F)c1', 'Clc1cc(/C=Cc2n[nH]c(-c3ccc([N+](=O)[O-])cc3)n2)ccc1N1CCOCC1', 'Clc1cccc([S@@]c2nc3ccccc3c(=O)n2C[C@H]2OCCO2)c1', 'Clc1ccccc1NS(=O)(=O)c1ccccc1N[C@H]1CCSc2ccccc21', 'Oc1ccc([N+](=O)c2ccccc2)cc1Br', 'ClCc1noc(-c2ccco2)n1', 'Clc1cccc(Br)c1O/C=C/C(=O)c1ccc(Cl)cc1Cl', 'ClCc1cc(Br)cnc1OCCn1cccnc1=O', 'Oc1ccc([N+](=O)CNc2ccccc2OCC2CCCCC2)cc1', 'Clc1ccccc1NCC[NH2+]Cc1ccccc1', 'Clc1ccc(Cc2noc(/N=C/C=C/c3ccccc3)n2)cc1Cl', 'Clc1ccc(-c2cc(F)c(F)c(F)c2COc2ccc(CC(=O)[O-])cc2)cc1', 'O=C(Cc1c[nH]cc1Cl)NCc1nc(C2CCCCC2)no1', 'Clc1cccc(Br)c1CO/N=C1\\C(=O)Nc2ccccc21', 'Clc1cc(Br)c2nc(Oc3ccccc3F)[nH]c(=O)c2c1', 'O=C(CCOc1ccc(Cl)cc1)N(Cc1cccc(F)c1)C(=O)c1nc2ccccc2c(=O)[nH]1', 'NS(=O)(=O)N(C)c1ccccc1OCc1nc2ccc(Br)cc2[nH]1', 'O=C(/N=c1\\occc(-c2ccccc2)n1)c1cccc(F)c1', 'Clc1ccccc1NS(=O)(=O)NC1CCN(O)CC1', 'O=C(c1cnc(N[C@H]2CCOC2)cn1)Nc1ccc(Cl)c2ccccc12', 'ClCc1csc(OCC(=O)c2cccc(Cl)c2Cl)n1', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(F)cc1NCc1ccco1', 'Nc1n[nH]c(-c2ccc(F)cc2)c1C(=O)Nc1ccc2c(c1)COC2', 'O=C(CNS(=O)(=O)C1CCCCC1)NNc1cc(C#N)ccc1O', 'ClCc1cc(CNc2cccc(C(F)(F)F)c2)n[nH]1', 'Clc1ccc(-c2cccnc2)nc1C[C@@H](O)COc1ccccc1Br', 'Oc1cc(Br)ccc1OCc1ccccc1OC[C@@H]1CCCO1', 'Clc1ccc(-c2nonc2)nc1COCc1ccccc1[N+](=O)[O-]', 'ClCc1n[nH]c(-c2ccccc2)c1N1CCc2cc(F)ccc2C1', 'Clc1cccc(Br)c1SCC(F)F', 'Clc1cc(Nc2nc3ccccc3[nH]c2=O)cnc1Sc1ccc(F)cc1Cl', 'Clc1cc(Nc2ccccc2)sc1-c1ccc(Cl)s1', 'Cc1ccc2nc(NC(=O)c3ccc(C(=O)[O-])cc3)sc2c1C', 'Clc1ccccc1C1(C[NH2+]Cc2ccccc2)CCCC1', 'Clc1ccsc1/N=C/c1cccc(COc2cccnc2)c1', 'Clc1cc(Cl)nc(Cl)c1Cl', 'Clc1ccc(C2SCCCC2)cc1', 'Clc1ccc(-c2nonc2N2CCC[C@H](C(=O)Nc3ccc4c(c3)CCC4)C2)cc1', 'O=C(c1cc(Cl)c[nH]1)N1CCc2ccc(NN)cc21', 'Nc1[nH]nc(Cn2cc(-c3ccc(Cl)cc3)cn2)n1', 'Clc1ccccc1N1N=C(c2ccco2)CC1', 'Clc1ccc(-c2csc3scnc32)cc1', 'Clc1ccc(-c2csc3scnc23)cc1', 'Clc1ccc(-c2csc3scnc23)cc1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'Clc1ccc(-c2csc3s/cnc23)cc1', 'O=C(c1cnc(/N=C/CCO)cc1)OCc1nc(-c2ccncc2)no1', 'COc1nnnn1CCNC(=O)c1ccc(Cl)c([N+](=O)[O-])c1C', 'COc1nnnn1/N=C/c1ccc(C)cc1', 'Clc1ccc(Br)cc1-c1cn(-c2c(F)cccc2F)nc1N1CCCCC1', 'Oc1cccc2ncoc12', 'Oc1cccc2ncoc12', 'Clc1cccc2cc(Br)[nH]c(=O)c12', 'O=C(CN(c1cn[nH]n1)S(=O)(=O)c1cccc(Br)c1)N1CCN(c2ccccc2)CC1', 'Clc1cc(/C=C(\\F)C(F)(F)F)ccc1Cl', 'ClCc1n[nH]c(Nc2ccccc2Cl)n1', 'Clc1nnnn1c1cc(F)ccc1F', 'Clc1nnnn1-c1cc(F)ccc1F', 'O=C(CCc1cc[nH]n1)Nc1ccc(Cl)cc1O', 'CNC(=O)c1cc(C(=O)N[C@H](C)c2cc(Cl)ccc2OC)ncn1', 'Clc1nnnn1-c1cc(F)ccc1Cl', 'Clc1ccc(-c2cccc(C[NH2+]COc3ccc(F)cc3)c2)cc1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1O', 'O=[N+]([O-])c1ccc2[nH]ncc2c1O', 'COCCC(=O)N1CCc2ccc(-c3ncoc3C)cc21', 'O=C(COCCNC(=O)N(C1CC1)c1ccccc1)OC[C@@H]1CCCO1', 'Clc1ccccc1OCc1csc(C(=O)[O-])n1', 'ClCc1n[nH]c2ccc(F)cc12', 'Clc1ccccc1C1CC[C@H](c2ccccc2F)C1', 'Clc1ccc(-c2nonc2Cn2cncn2)cc1', 'ClC[C@H](CO)Nc1ccc(C#Cc2ccccc2)cc1', 'Clc1ccc(-c2nonc2N2CCN(Cc3cc(Cl)cs3)CC2)cc1', 'Oc1ccccc1[C@@H]1CCCN1C(=O)NCCCCn1cncn1', 'O=[N+]([O-])c1c(-c2cc(-c3ccccc3)on2)sc2ncccc12', 'Clc1ccccc1NS(=O)(=O)NC[C@@H]1CCCO1', 'O=[N+]([O-])c1c(-c2ccccc2)sc(SCC(=O)N2CCOCC2)n1', 'O=[N+]([O-])c1cccc2c1nc([O-])c1ccccc12', 'O=[N+]([O-])c1cc(Cl)c(S(=O)(=O)N/N=C/c2ccc(Br)cc2F)cc1Cl', 'Clc1cccc2cc(F)nnc12', 'Clc1ccc(-c2csc3[nH]c4cc(Br)cnc4c32)cc1', 'Clc1nnnn1-c1c(Cl)cccc1Cl', 'Clc1nnnn1-c1c(Cl)cccc1Cl', 'Clc1ccc(Br)cc1-c1cnc2ccc(F)c(Br)c2c1N', 'Nc1n[nH]c(-c2cccc(F)c2)c1NC(=O)c1ccc(S(=O)(=O)C(F)F)cc1', 'O=[N+]([O-])c1cccc2nc(-c3cc(Cl)ccc3[N+](=O)[O-])[nH]c12', 'OCS[C@H](CCc1ccccc1)c1ccc2c(c1)OCO2', 'O=[N+]([O-])c1c(-c2ccccc2)n(-c2ccccc2F)nc1[O-]', 'Clc1ccsc1[C@H](c1ccccc1)N1CCOCC1', 'CNS(=O)(=O)c1cccc(C(=O)Nc2nc(C3CC3)cc(C)c2[N+](=O)[O-])c1', 'O=C(CSCN1C(=O)/C(=C/c2ccccc2)SC1=S)NCc1ccc2c(c1)OCO2', 'O=C(Cc1c(O)cccc1Cl)N1CCC[C@H](O)C1', 'Clc1ccc(Cc2nnc(NC[C@H]3CCCO3)o2)cc1', 'Oc1cc(-n2cc(C(F)F)ccc2=O)ccc1C(=O)Nc1cc(F)ccc1[N+](=O)[O-]', 'O=C(CSC(=O)Nc1ccc2c(c1)nc1ccccn12)N1CCC(Cc2ccccc2)CC1', 'Nc1c(F)c(Br)ccc1F', 'Clc1ccc(-c2csc3[nH]cnc32)cc1', 'Clc1ccc(-c2csc3[nH]cnc23)cc1', 'Clc1ccc(-c2csc3[nH]cnc23)cc1', 'Oc1cccc(S[C@@H](c2nccn2-c2ccccc2)c2ccccc2)c1', 'O=[N+]([O-])c1ccncc1[C@@H](O)c1ccccc1', 'Clc1cc(C(F)(F)F)nc(-c2cccc(F)c2Cl)n1', 'Oc1ccccc1[C@@H](NS(=O)(=O)c1ccc(F)cc1)c1cc(F)ccc1F', 'Clc1ccc(-c2cc(O[C@H]3CCCOC3)nc(-c3ccc(Cl)cc3)n2)cc1', 'O=C([C@H](CCC1CC1)c1c[nH]c2ncccc12)N1CCCC1', 'O=C(/N=C/c1ccc[nH]1)Nc1ccc(-c2ccc(F)cc2)nc1', 'Clc1ccccc1NS(=O)(=O)NC[C@H]1CCCCO1', 'O=[N+]([O-])c1cc(Br)c(S(=O)(=O)Nc2ccc(F)cc2)c2c1CCO2', 'Clc1cc(NSc2nnc(-c3ccc(Cl)cc3)n2Cc2ccc(F)cc2)ncn1', 'Nc1[nH]nc(Nc2ccccc2F)c1[N+](=O)[O-]', 'Clc1ccccc1Nc1nccn1/C=O', 'O=C(c1ccsc1CN1C(=O)CCCC1=O)Nc1cc(C(F)(F)F)ccc1F', 'Oc1cc(-n2nc(Cl)cc2C(=O)c2cc(Cl)ccc2Cl)ccc1O', 'O=[N+]([O-])c1cc(S(=O)(=O)CCCc2cn3ccccc3n2)ccc1F', 'O=[N+]([O-])c1c(-c2cc(-c3ccc(F)cc3)n[nH]2)ncnc1C1CC1', 'O=C(C1=CN(c2cccc(Oc3ccccc3)c2)CCC1)N1CCc2nc[nH]c2C1', 'Oc1ccccc1/N=C/c1cccc(OCC(F)F)c1O', 'ClC(c1ccccc1)c1cccc(Cn2cncn2)c1', 'O=C([O-])/C(=C\\Cc1ccccc1)n1nnc2ccccc21', 'Oc1ccc([C@H](Nc2cc(C(F)(F)F)c3ccccc3n2)c2ccncc2)cc1', 'Clc1cc(/N=C/c2c(Cl)cccc2Cl)nc2ccccc12', 'O=[N+]([O-])c1ccnc2c1nnn2-c1cccc(C(F)(F)F)c1', 'O=C(c1ccsc1)[C@@H]1CCCN(Cc2ccccc2F)C1', 'Cc1cn(CC(=O)N(CCN2CCOCC2)Cc2ccccc2F)nc1[O-]', 'Oc1ccccc1O[C@@H](c1ccccc1)c1ncccc1O', 'COc1ncnc(CNc2cccc(OC(C)C)c2)c1[N+](=O)[O-]', 'O=C(CS(=O)(=O)c1ccc(Br)cc1)Nc1cc(Br)on1', 'O=C([C@H](CNc1ccc(C(F)(F)F)cc1)c1cccs1)NNC(=O)c1ccc2c(c1)OCO2', 'O=C(/N=C/CCSc1ccccc1)c1ccco1', 'Clc1ccccc1Cn1nnnc1SCn1cccn1', 'Oc1cccc2cc(COc3cccc(C[N+](=O)[O-])c3)[nH]c12', 'O=C(CC1([O-])CCC1)NCCc1nc2ccccc2o1', 'O=C(CNCc1nc2ccccc2[nH]1)Nc1cc(C(=O)[O-])cc(Cl)c1', 'OCc1ncn(-c2ccc(Cl)cc2Cl)n1', 'O=[N+]([O-])c1ccc(Oc2cc(C3CCCCC3)co2)cc1', 'O=[N+]([O-])c1cc(Cl)c2nc(N)sc2c1', 'O=[N+]([O-])c1c(Cl)nccc1C1CC1', 'Clc1ccccc1OC1CN(C(=O)c2cc(OCc3ccco3)on2)C1', 'Clc1ccc(Br)c([C@@H](O)c2cccnc2)c1', 'Clc1cccc(CCNC(=O)N[C@@H]2CCC[C@H](C3CC3)C2)c1', 'Clc1cccc(Br)c1O[P@](=O)(=O)c1ccccc1', 'Oc1ccc(F)c(F)c1NCCC(F)(F)F', 'Clc1ccccc1CSC/C=C/C(=O)[O-]', 'O=C(CSCc1ccco1)N/N=C/c1ccc(Oc2cc([N+](=O)[O-])ccc2Cl)cc1', 'Oc1cc(-n2cnnn2)cc(-c2ccco2)c1', 'Clc1ccccc1-c1cscc1-n1cncn1', 'O=C([O-])/C(=C/c1ccccc1)NC[C@@H]1CCCO1', 'Clc1ccc(CCNC[C@@H]2CCOc3ccc(F)cc32)cc1', 'Oc1cccc2ncn([C@@H](Cc3ccccc3)c3ccccc3)c12', 'Oc1ccc([N+](=O)c2cccc(C(F)(F)F)c2)cc1', 'Oc1cccc(Nc2snc3ccccc23)c1F', 'Clc1cccc([S@@]c2c[nH]nc2)c1', 'ClCc1ccnn1Cc1ccc(F)cc1O', 'Oc1cc(F)nc(SCC(=O)Nc2ccc(S(=O)(=O)C(F)F)cc2)c1', 'O=C(/C=C/O)C(=O)NCc1ccccc1Br', 'Oc1ccc(F)cc1NSc1ccc2nnc(-c3ccccc3)n2c1', 'Clc1cc(/N=C/c2nc3ccccc3o2)ccc1Oc1ccc([N+](=O)[O-])cc1[N+](=O)[O-]', 'O=[N+]([O-])c1cc(Cl)c2[nH]ncc2c1', 'Clc1ccc(Br)c(C(=O)/C=C/C2=CCCCC2)c1', 'Clc1cc(F)cc(I)c1O', 'ClC(=S)Oc1ccc(Cl)cc1Br', 'Clc1cc(NSc2nncn2C2CC2)ccc1Oc1ccccc1', 'NS(=O)(=O)/C(=C(\\C)c1csc(NC(=O)c2ccc(F)cc2)n1)c1ccccc1', 'Oc1ccc([C@H]2NC(=O)c3ccccc32)c2ccccc12', 'Clc1ccccc1O/C=C(/C#N)C(=O)N1CCCC1', 'CN[C@H](C1(CCC2CC2)CC1)c1ccc(C)cc1', 'Clc1cc(Br)c2nc(-c3ccco3)cc(-n3ccnc3)c2c1', 'Oc1cc(CCNc2nc3ccccc3c(=O)n2-c2ccccc2)ccc1SCC(F)(F)F', 'Oc1ccc(Oc2cc(C(=O)[O-])cc([N+](=O)[O-])c2)cc1', 'N#Cc1ccc(-c2n[nH]cc2-c2cccc(C(F)(F)F)c2)cc1', 'OC[C@H]1C(=O)C[C@H](C(=O)N(Cc2ccc(Cl)cc2)c2ccccc2)C1', 'CNS(=O)(=O)[C@@H](C)Cc1ccc([N+](=O)[O-])cc1', 'ClCc1ccc2nnc(-c3ccc(Br)c(F)c3)n2n1', 'Oc1ccc(O[C@@H](C(=O)[O-])c2ccccc2)cc1', 'Clc1cccc2cc(CNC[C@@H]3CCc4ccccc4O3)c(F)cc12', 'O=C(/N=C\\Nc1cccc(S(=O)(=O)N2CCOCC2)c1)C1CCC(c2ccccc2)CC1', 'Clc1cc(Br)cc(F)c1C(=O)Nc1nccc(-c2cnccn2)n1', 'CNC(=O)N[C@@H](C)c1ccc(F)cc1F', 'Oc1cc(-n2cc(Br)cn2)ccc1C(F)(F)F', 'Clc1cc(Br)c([C@H](O)c2ccccc2)cc1F', 'NS(=O)(=O)Cc1ccc(NC(=O)[C@H](O)c2cccc(Cl)c2)cc1', 'Oc1ccc(/C=C/N(CC(F)F)c2ccccc2F)cc1', 'Clc1ccccc1Oc1cc(NC2(C(=O)[O-])CCOCC2)ccc1F', 'NS(=O)(=O)/N=C/C(=O)Nc1ccc(Cl)cc1', 'ClCc1cccc(Br)c1[N+](=O)[O-]', 'Clc1ccc(-c2nnnn2[C@@H](Cc2ccccc2)c2noc(C3CC3)n2)cc1', 'Clc1ccc([C@@H](OC[C@@H](O)C2CC2)N2CCN(c3ccccc3)CC2)cc1', 'Cc1nc(C(C)C)c2nc(Nc3ccc(Cl)cc3)sc2n1', 'O=C(COc1cc([N+](=O)[O-])c([O-])cc1Br)NCc1ccc(Cl)cc1', 'Clc1ccc(Br)cc1S[C@H]1CCCc2ccccc12', 'O=[N+]([O-])c1c(-c2ccoc2)cccc1[N+](=O)[O-]', 'Oc1cccc([C@H]2C[C@H]2C(=O)N2CCC(OCc3ccccc3)CC2)c1', 'Clc1nnnn1-c1ccc(Br)cc1F', 'N[C@@H](NC(=O)c1ccccc1OCc1ccco1)C1CCC1', 'O=[N+]([O-])c1ccc2oc(SCc3ccc(Br)o3)nc2c1', 'Clc1ccccc1Oc1cc(Cl)c2[nH]nnc2c1', 'ClCc1cc(Br)c2c(c1)OCO2', 'Clc1ccc([C@@H](OC)C(=O)N2CCC[C@@H](Cn3cc4ccccc4n3)C2)cc1', 'Nc1c(F)c(F)cc(F)c1F', 'NS(=O)(=O)/C=C/C=C/c1ccccc1', 'O=C(/N=C/c1ccc([N+](=O)[O-])cc1)C(F)(F)c1ccccc1', 'NS(=O)(=O)N[C@@H]1CCN(c2ccccc2F)C1', 'O=C(Cc1n[nH]n1C1CC1)Nc1ccc2oc(-c3cccs3)nc2c1', 'Nc1nc(-c2ccc(Cc3cccs3)nc2)ccc1C(F)(F)F', 'Clc1cccc(SC[C@@H](O)c2ccc(Cl)cc2F)c1', 'Clc1ccc(CO)c(-c2cc(Cl)cn3ncnc23)c1', 'O=[N+]([O-])c1ccc2[nH]cc(-c3ccn[nH]3)c2c1', 'COc1nn(-c2ccccc2)c(C(C)C)c1NS(=O)(=O)c1cccc(C(F)(F)F)c1', 'Clc1cccc([S@@]C2=NCCc3ccccc32)c1', 'O=C(/C=C1/NC=CCC1)Nc1ccc(Cl)cc1Cc1ccccc1', 'Nc1nncn1CCCc1nc(-c2ccccc2F)cs1', 'O=C(CC1CC1)N1CCC(CC(=O)N(CCO)C2CC2)CC1', 'Cc1ccsc1CN1CCO[C@H](c2cccc([N+](=O)[O-])c2)C1', 'O=C(/N=c1/occn1Cc1ccccc1)c1cccc(F)c1', 'Clc1ccc(-c2n[nH]cc2-c2ccc(S(=O)(=O)C(F)(F)F)cc2)cc1', 'OCc1n[nH]c(Nc2ccc(Br)cc2)n1', 'NS(=O)(=O)/C=C\\c1c(Cl)cccc1Cl', 'Clc1ccc(Br)cc1/N=N\\c1ccccc1Br', 'Oc1cccc(-c2cc3occc3nc2NC2CCC(CO)CC2)c1', 'CN[C@H](c1ccc(Cl)cc1Cl)c1cc(F)ccc1F', 'Clc1ccccc1CO[C@H]([NH2+]Cc1ccc(F)cc1)c1ccccc1F', 'Clc1nncc(OCc2cccc([N+](=O)[O-])c2)n1', 'Clc1ccc(CN[C@@H](C[C@@H]2CCCCO2)c2nccn2-c2ccccc2F)cc1', 'O=C(c1ccsc1)N(c1ccc2ccccc2c1)[C@@H]1CCO[C@@H]1c1ccc(Cl)cc1F', 'O=C(CNCc1ncoc1Cc1ccncc1)Nc1ccc(Cl)cn1', 'Oc1ccc([N+](=O)[O-])c(NN=C2CCCCC2)c1', 'Oc1cc(Cn2ncc(Cl)c2[N+](=O)[O-])cnc1Cc1ccccc1', 'Oc1cc(Br)c2sccc2c1OCc1ccc(O)cc1', 'O=C(/C=C1CC(=O)c2ccccc2[C@@H]1c1c[nH]c2ccccc12)c1cccc(Cl)c1', 'Clc1cc(Nc2n[nH]nc2C2CC2)ccc1Cl', 'O=C(Cc1cc2occc2[nH]1)NNC(=O)c1cc(Cl)ccc1Cl', 'O=C(CSCc1csnn1)NCc1ccc(-c2nc3ccccc3o2)cc1', 'Clc1ccc(-c2csc3[nH]cc(C(F)(F)F)c-23)cc1', 'Cc1sc(C(=O)N/N=C/c2ccc(C(C)C)cc2)c(C#N)c1C', 'O=C([C@H](Sc1n[nH]c2ccccc12)c1ccccc1)N(Cc1ccc(Cl)cc1)C1CC1', 'O=[N+]([O-])c1ccc2oncc2c1', 'O=C(CSCNOc1ccccc1Cl)Nc1ccc(O)cc1', 'Clc1cccc(-c2noc(CSc3ncnc4[nH]ncc34)n2)c1', 'Clc1ccc(Br)cc1OCC[C@@H]1CNC(=O)N1Cc1ccccc1', 'NS(=O)(=O)/C=C/c1cccnc1', 'Clc1ccc(-c2n[nH]cc2CN2CC[C@@H](Nc3ccccc3F)C2)cc1', 'CC[C@@H](COC)S(=O)(=O)c1ccc(OC)c(OC)c1', 'Oc1cc(C[NH2+]Cc2ccc(F)cc2)ccc1Oc1ccccc1', 'Clc1cc(Br)c([N+](=O)[O-])cc1Br', 'COc1nnnc(Nc2c(Cl)cccc2Cl)c1C', 'O=C(/N=c1/[nH]c2ccccc2c(=O)n1-c1ccccc1)c1csc(C2CC2)n1', 'O[C@@H]1[C@@H](c2ccccc2)Oc2ccccc21', 'Nc1nonc1CN1CCN(C(=O)/C=C/C2CC2)CC1', 'C=CCn1cc(NC(=O)c2cc(C(C)C)ncc2F)cn1', 'O=C(/N=C/C(=O)Nc1ccc(OC(F)(F)F)cc1)c1ccc([N+](=O)[O-])cc1', 'Clc1ccc(-c2cc(Br)co2)cc1F', 'O=[N+]([O-])c1cc(C2CCCCC2)n2ncc(Cl)c2c1', 'CN(CC(=O)N[C@H]1CCN(c2ccccc2)C1=O)C(C)C', 'N#Cc1c(F)cccc1NC(=O)N[C@H](CO)Cc1ccccc1', 'O=[N+]([O-])c1cc(Cl)nc2cnccc12', 'Oc1ccc([N+](=O)c2ccc(Br)cc2)cc1Cl', 'O=C(Cc1nn(-c2ccccc2)c([O-])c1Cl)Nc1cccc([N+](=O)[O-])c1', 'Clc1cccc2ncn(CCCC(=O)Nc3nncs3)c12', 'CNS(=O)(=O)[C@@H]1CCN(Cc2cccc(OC)c2)C1', 'Clc1ccccc1[C@H](c1nc2ccccc2s1)N1CCc2ccccc2C1', 'Clc1nnnn1-c1cnccc1', 'Clc1nnnn1-c1cnccc1', 'Clc1ccsc1Cn1c(=O)n(Cc2ccco2)c2ncccc21', 'ClCc1nc(N2CCCCC2)c2nc(N3CCCCC3)nnc2n1', 'Clc1ccccc1CSc1cncc(OCC(F)F)n1', 'O=C([C@H](c1ccccc1)n1cccc1)c1cccc(Cl)c1', 'COc1ncncc1C(=O)Nc1cc(S(=O)(=O)C(=O)N2CCCC2)ccc1F', 'Oc1ccc(/N=C/c2cccc(C(F)(F)F)c2)nc1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccn1Cc1cccc(Cl)c1', 'Nc1c(Oc2cc(Cl)ncn2)cnn1-c1ccc(F)cc1', 'Clc1ccc(Br)cc1-c1cnc(C2CC2)nn1', 'O=[N+]([O-])c1cc(Cl)n(-c2ccccc2F)n1', 'Oc1ccc(F)cc1Cc1c[nH]c(Nc2ccccc2Cl)n1', 'Clc1ccccc1Oc1cc(-c2cccs2)n[nH]1', 'OCc1n[nH]cc1CNCc1ccc([N+](=O)[O-])cc1', 'COc1nn(C)cc1CNC(=O)N(CCS(C)(=O)=O)c1ccccc1', 'ClC(=S)c1ccc(N[C@@H]2CCc3c(Cl)cccc32)cc1', 'COc1nnnc(NC(=O)CNc2ccc(Cl)c([N+](=O)[O-])c2)c1C', 'Clc1ccc([C@@H](NS(=O)(=O)c2ccc(F)cc2)NC(=O)C2CC2)cc1', 'Oc1cc(Cn2cnc3ccccc32)c2[nH]ccc2n1', 'NS(=O)(=O)/C=C/CSc1ccc(F)cc1', 'Clc1ccc(-c2nonc2C2CC2)cc1F', 'ClC(=S)NNc1ccccc1[N+](=O)[O-]', 'Clc1cc(Br)c(COC2CC2)cn1', 'Clc1cccc([S@@]c2cc(I)cnc2Cl)c1', 'OC[C@H](Cl)c1ccc2ccccc2c1', 'N=Cc1cc([N+](=O)[O-])c(Cl)cc1Oc1ccccc1Cl', 'Clc1ccc(Br)c([C@@H](CO)Nc2ccc(F)cc2)n1', 'Oc1ccc(/C=N/O[C@@H](c2ccc(F)cc2)C2CC2)cc1', 'NS(=O)(=O)C[C@H](Cc1c[nH]c2ccccc12)NC(=O)c1ccccc1[N+](=O)[O-]', 'Clc1cc(/C=N/n2cc(S(=O)(=O)Cc3ccccc3)ccc2=O)ccc1C1CC1', 'O=C([C@H](c1c(Cl)cccc1Cl)c1ccc(Cl)cc1)c1ccccc1[N+](=O)[O-]', 'Clc1ccc(Cc2noc(-c3ccc(Cl)o3)n2)c(F)c1', 'N=Cc1cnc(Cl)nc1-c1ccc(Cl)cc1', 'Clc1ccsc1Sc1nnc(-c2ccccc2)n1C1CCCC1', 'O=[N+]([O-])c1c(Cl)nc(-c2ccc(OS(=O)(=O)C3CCCC3)cc2)c2ccccc12', 'Cc1ncsc1C(=O)N1CC=C(c2ccc(Cl)cc2Cl)CC1', 'Nc1nnc(-c2cccc(F)c2)cc1CN1CCC[C@@H](c2nc3ccccc3[nH]2)C1', 'CNS(=O)(=O)[C@@H](C)c1ccccc1', 'O=C(/N=C/N=C/c1cccc(Cl)c1)NCCOc1ccc(F)cc1', 'O=C([C@H](Cl)CBr)c1ccccc1', 'Nc1cnn(CCn2cc(Cl)c(=O)n(-c3ccccc3)c2=O)c1O', 'O=C(c1cc(F)c2c3ccccc3[nH]c(=O)n12)N1CCOCC1', 'O=[N+]([O-])c1c(Cl)nn(C[C@@H](O)COc2cccc(OCc3ccccc3)c2)c1-c1ccc(Cl)cc1', 'O=C(/C=C/c1cccs1)NNC(=O)Oc1ccccc1C(F)(F)F', 'NS(=O)(=O)/C=C/CSc1ccccc1', 'C=CC#CCN(C)C(=O)C1CCN(C(=O)c2cccc(F)c2)CC1', 'Clc1cccc(S[C@H]2CCOc3ccccc32)c1', 'O=C(/C=C\\c1n[nH]c(=O)[nH]1)c1ccc2ccccc2c1', 'O=C(c1c[nH]nc1-c1ccc(Cl)cc1)N(CC1CC1)S(=O)(=O)c1ccc(Cl)cc1', 'Nc1c(F)c([N+](=O)[O-])ccc1-n1cccc1', 'Clc1cc(F)cc2nc[nH]c12', 'Clc1ccc(Br)c(O)c1O[C@@H]1CCCC[C@@H]1OCCc1ccccc1', 'O=[N+]([O-])c1ccncc1[C@@H](OCC(=O)N1CCCC1)c1ccccc1', 'Nc1[nH]nc2ncccc12', 'Nc1[nH]nc2ncccc12', 'O=[N+]([O-])c1ccc(CCNC2=Cc3ccccc3CS2)cc1', 'O=C(Cc1ccco1)NC[C@H](CC(F)(F)F)c1ccccc1', 'Clc1ccc(-c2ccccc2)nc1CN[C@H]1CCCC[C@H]1CO', 'Clc1ccccc1-c1n[nH]cc1C(=O)CN1C(=O)COc2ccc(F)cc21', 'O=C(CCS)C1=Cc2ccccc2OC1', 'Clc1ccsc1/N=N\\N1CCN(c2ccccc2F)CC1', 'O=[N+]([O-])c1c(Cl)nnc2nc(NCc3ccccc3)[nH]c(=O)c12', 'NS(=O)(=O)/C=C\\c1cc(Cl)cc(Cl)c1', 'NS(=O)(=O)Cc1nnn(-c2ccccc2)c1N', 'COc1nc(NC(=O)Cc2c[nH]c(C)c2C)cc(NCc2ccccc2)c1C', 'ClCc1cccc(I)c1O', 'O=C(CSCNSc1ccccc1)c1ccccc1F', 'Oc1ccc([N+](=O)Cc2nc(C3CC3)no2)c2ccccc12', 'O=C(/C(c1cccs1)=[NH+]/C[C@H]1CC=Cc2sccc21)N1CCN(c2ccccc2)CC1', 'Clc1cccc2cc(-c3noc(C4CC4)n3)nnc12', 'Clc1cc(Cl)nc2ncnc(Nc3ccc4c(c3)OCO4)c12', 'Oc1cc(-n2cnc(CN3CCOCC3)n2)ccc1F', 'Oc1cc(F)c(-c2cccnc2)nc1CO', 'Clc1cccc(NC2(c3cccs3)CC2)c1F', 'Clc1cc(NSc2cccs2)ccc1OS(=O)(=O)c1ccccc1', 'O=C(CN(c1ccccc1)S(=O)(=O)c1ccc(F)cc1)NC[C@@]1(O)CCc2ccccc21', 'Clc1nnnn1Cc1cc(Cl)cc(Cl)c1', 'Clc1cc(N2CCO[C@@H](c3ccccc3F)C2)nc2ccccc12', 'Clc1ccsc1S(=O)(=O)c1cc(Cl)cc(Cl)c1', 'Clc1cccc(Br)c1N[C@@H]1CCc2ccccc21', 'Oc1cccc([C@H](O)Cc2nc3ccccc3s2)c1', 'Oc1ccccc1[C@@H](Cl)C1CC1', 'Clc1cc(/C=N\\OC(=O)COc2ccccc2Cl)c2c(c1)OCO2', 'ClCc1cc(C(F)(F)F)nc(N2CCOCC2)n1', 'COc1nc(C)cc(C)c1NC(=O)C(=O)Nc1ccc2[nH]c(=O)[nH]c2c1', 'O=C([C@H](Sc1ccccn1)C1CC1)N1CCN(c2ccccc2F)CC1', 'Clc1cc(Cl)nn1CCN1CCOCC1', 'Clc1ccc(-c2cc[nH]n2)c(Cl)c1', 'Clc1ccsc1-c1cncc(Br)c1', 'ClCc1cc([N+](=O)[O-])ccc1I', 'Clc1ccsc1NC(=O)Cc1c[nH]nc1Cc1ccccc1', 'Clc1ccccc1-c1n[nH]c2cnn(Cc3cccs3)c12', 'Oc1ccc(F)c(CCN2CC(=O)Nc3ncccc23)c1', 'OCSc1nc(-c2ccco2)nc2cc(Cl)ccc12', 'Clc1ccc(Br)cc1Sc1ncn[nH]1', 'CNS(=O)(=O)c1c(OCC)nn(C)c1C', 'Cc1snnc1S(=O)(=O)c1ccc(S(=O)(=O)N(C)C)cc1', 'Clc1cc(Cl)nnc1OCc1ccco1', 'Oc1ccc([N+](=O)CN(C(=O)c2cccc([N+](=O)[O-])c2)c2ccccc2)cc1', 'Clc1ccccc1-c1nc(CS(=O)(=O)N2CCCCC2)c[nH]1', 'O=[N+]([O-])c1c(Cl)nsc1-c1ccccc1', 'Clc1ccsc1Sc1ncccc1C(=O)Nc1ccc(F)c(Cl)c1', 'Clc1cccc(Nc2cc(N[C@H]3CCOC3)nc(-c3ccccc3)n2)c1', 'ClC(=S)Nc1ccc(Sc2ncccc2Cl)cc1', 'Clc1nncn1CCN[C@H](c1ccccc1)c1ccccc1F', 'O=[N+]([O-])c1c(Br)cnc2cccnc12', 'O=C(CNC(=O)c1ccc2c(c1)N(C(C)C)CCC2)NCC(=O)[O-]', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(SCCCO)n1', 'NS(=O)(=O)/C=C/COc1ccc2cccnc2c1', 'O=C([C@H](N[C@@H]1CCc2ccccc21)[NH+]1CCCC1)c1ccc2c(c1)OCCO2', 'O=[N+]([O-])c1c(Cl)nc2ccccn12', 'Oc1ccc(F)cc1-n1cccc1CO/C=C/c1ccccc1Cl', 'Clc1ccc(Br)c(Cl)c1C(=O)N1CCc2ncc(-c3cccc(Cl)c3)nc2C1', 'N#Cc1c(O)cc(N)cc1C(=O)Nc1cccc(OCO)c1', 'Clc1ccccc1[C@H](N[C@H]1CCSc2ccc(Cl)cc21)c1ccccc1', 'Oc1ccc(/C=N/N=C2\\C(=O)N(c3ccccc3)C(=O)NC2=S)cc1', 'ClCc1csc(Nc2ccc(OC(F)F)cc2)c1', 'Oc1ccc(F)cc1N1CCOC[C@H]1C(=O)N(Cc1ccccc1)C1CC1', 'Clc1cc(F)c(O)c(F)c1F', 'O=C(COCCOc1ccc(Br)cc1)Nc1nc([O-])c(Cl)c2[nH]ccc12', 'O=[N+]([O-])c1cc(Nc2ncccc2F)ccn1', 'O[C@@H](Sc1nnc(-c2ccccc2)o1)C(=O)Nc1ccccc1', 'O=C(/C=C/c1cccc([N-]S(=O)(=O)c2ccc(Cl)cc2)c1)N(Cc1ccccc1)C1CC1', 'NS(=O)(=O)/C=C(\\c1ccccc1)N1CCN(Cc2ccc3c(c2)OCCO3)C1', 'Clc1ccccc1O/N=C\\Nc1c(Cl)cccc1Cl', 'Clc1nnc(SCc2ccc(F)cc2F)s1', 'COc1nc(C)c(C(=O)O[C@H](C)C(=O)N2CCN(c3ccc(C)cc3)CC2)cn1', 'O=C(COCCCc1cccc(Cl)c1)N[C@@H]1CCc2c(O)cccc21', 'NS(=O)(=O)/C(=C\\Cc1cccc(C(F)(F)F)c1)c1cccc2ccccc12', 'Oc1cc(-n2nc3c(c2)CCCN3C(=O)c2ccc(Cl)cc2)cc(Cl)c1', 'Clc1ccccc1Nc1cc(OC[C@H]2CCCO2)ncn1', 'Clc1cc(Cl)c(Cc2cccs2)c2ncc(-c3ccccc3)n12', 'COc1nc(S(=O)(=O)CC(C)C)ccc1C', 'Oc1ccc(/C=C(/C#N)c2cnc(Cl)cn2)cc1', 'COc1ncnc(NC(=O)[C@@H](C)n2c(=O)n(Cc3ccc(F)cc3)c(=O)c3ccccc32)c1C', 'Clc1cc(F)c(CN2CCC[C@@H](Cc3cccc4cccnc34)C2)cc1', 'Clc1ccc(Br)cc1OS(=O)(=O)C1CC1', 'Oc1ccc([C@H]([O-])CCOc2ccc(Cl)cc2)cc1', 'Oc1ccccc1-c1n[nH]c(Nc2cc(F)c(Cl)cc2Cl)n1', 'N[C@@H](Sc1ccccc1)c1ccccc1[N+](=O)[O-]', 'COc1nn(CC)c2ncc(C(=O)c3cc4ccccc4o3)nc12', 'Oc1ccc(F)c(F)c1SCc1nc2ccccc2c(=O)[nH]1', 'Clc1ccccc1N1Nc2ncccc2C1', 'O=[N+]([O-])c1c(Br)sc(CN2CCN(c3cccc(Cl)c3)CC2)c1CO', 'OC[C@H](C(=O)N1CCN(c2nc(-c3ccccc3)cs2)CC1)c1cccs1', 'OCc1nn(-c2ccccc2)c(=S)n1C1CC1', 'Clc1ccsc1-c1cc(COc2cccc(Cl)c2Cl)on1', 'OC[C@H](CNS(=O)(=O)c1cccs1)c1ccccc1', 'Clc1ccc(-c2csc3[nH]ncc3c2=O)cc1', 'Clc1ccc(-c2cccnc2N=C2CCOCC2)cc1', 'O=[N+]([O-])c1ccncc1N1CCSc2ccccc21', 'CN[C@H](NC(=O)NC[C@H](O)[C@@H]1CCCO1)c1ccccc1', 'Clc1ccccc1Cn1ncc2c1CCN(C(=O)[C@@H]1COc3ccccc3C1)C2', 'O=C(COc1ccc(CN2CCOCC2)cc1)N[C@@H]1C[C@H]2CCCC[C@@H]2C1', 'Clc1ccccc1[C@H](CO)NCc1ccc([N+](=O)[O-])c([N+](=O)[O-])c1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(I)c1-c1ccc([N+](=O)[O-])cc1', 'O=C(C1=CC(=N/O)CC1)c1ccccc1', 'N=C1CSc2ccc(C(=O)NC3CCCC3)cc2N1', 'Clc1ccccc1[C@H](CCO)Nc1ccc2nccnc2c1', 'Clc1ccc(Br)cc1Cc1nnc(-c2ccc(Cl)cn2)n1N', 'O=C(c1cccn1c1ccc(Cl)cc1)N1CCOC[C@@H]1CO', 'O=[N+]([O-])c1c([N+](=O)[O-])c(-c2ccccc2)nc2sccc12', 'O=C([O-])/C(C[C@@H](O)c1ccccc1)N(Cc1nc2ccccc2n1Cc1ccccc1)C1CC1', 'O=C(CNc1ccc([N-]S(=O)(=O)c2ccccc2)c([N+](=O)[O-])c1)Nc1ccc(Cl)cc1Cl', 'O=C([C@H](c1c(Cl)cccc1Cl)C(F)F)N1CCN(Cc2ccco2)CC1', 'O=C(c1c[nH]nc1-c1cc(Cl)ccc1Cl)N1CCOCC1', 'Clc1ccc(Cc2csnn2)cc1NCCO', 'Oc1cccc(N2C[C@@H](C(=O)N3CCc4sccc4[C@@H]3c3ccccc3)CC2=O)c1', 'O=C(/C(CO)(c1ccccc1)c1cc2ccccc2o1)c1ccccc1', 'Clc1cccc2cc(F)[nH]c12', 'Clc1cccc2cc(F)[nH]c12', 'O=C(COc1cc(F)ccc1Cl)NN[C@H]1CCOc2ccccc21', 'Clc1ccsc1C(F)(F)Cc1ccccc1Cl', 'Clc1ccsc1-c1nnnn1-c1ccc(F)c(Cl)c1', 'CC(C)CS(=O)(=O)N(C)Cc1ccc(Br)o1', 'O=[N+]([O-])c1cccc2c1[nH]c(=S)n2CC1CCOCC1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc([N+](=O)[O-])c1-c1cccs1', 'O=C(/N=C/CCC#N)c1ccccc1', 'Oc1cccc2cc(/C=N/c3ncnc4[nH]ncc34)ccc12', 'Clc1cccc(F)c1[C@@H](CCn1cccn1)N[C@@H](c1ccccc1)c1ccccc1', 'Clc1ccsc1/CC(=O)N/N=C/c1ccc2cc(Cl)ccc2n1', 'Oc1ccccc1[C@@H]1COc2ccccc2N1CN1CCOCC1', 'Clc1ccc(Br)c(F)c1-n1cncn1', 'Clc1cccc(F)c1-c1ccc(Br)c(C(=O)[O-])c1', 'Oc1cc(Cn2c(=O)oc(C(=O)NCc3ccccc3)c2C2CC2)ccc1[O-]', 'O=C(/N=C/c1ccc(Cl)c2ncccc12)N1CCN(S(=O)(=O)c2ccccc2)CC1', 'NS(=O)(=O)N1Cc2ccc(OC(=O)c3cc(C#N)ccc3Cl)cc2C1', 'Nc1c(Oc2cccc(C(=O)N3CCOCC3)c2)cnn1N', 'Clc1ccccc1O/N=C(/[O-])Cc1cccc(OC2CCCC2)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])c(Br)cc(OCc2cccs2)c1C#CCOc1ccccc1', 'Clc1cccc(Br)c1N[C@@H](c1ccc(F)cc1)N1CCCC1', 'Clc1ccccc1-c1cncnc1NC[C@@H]1C[C@H]1c1ccccc1', 'O=C(COCC(F)(F)F)NOCc1cccs1', 'O=[N+]([O-])c1c(Br)cc(S(=O)(=O)N2CCOCC2)cc1OC(F)F', 'Clc1cc(Br)cc2[nH]cnc12', 'O=C(/C(Cc1ccc[nH]1)c1cccc(F)c1)c1ccc(Cl)cc1', 'Clc1ccccc1Nc1nc2sccn2c1Br', 'Clc1ccccc1O/N=[N+]=NC(=O)CN1CCN(c2cnccc2)CC1', 'Clc1ccc([C@@H](CC(=O)N2CCOc3ccc(Cl)cc32)N2CCOCC2)cc1', 'Clc1ccc(Br)cc1/N=N\\c1ccc(Cl)cc1[N+](=O)[O-]', 'Clc1cccc(/C(CC(=O)c2ccc(OCc3cnn(C)c3)cc2)N2CCCC2)c1', 'O=[N+]([O-])c1ccn(Cc2noc(-c3ccc(F)cc3F)n2)c1', 'NS(=O)(=O)C[C@@H](c1ccccc1)N1Cc2ccccc2C1', 'Clc1ccc(-c2n[nH]c(OCC(F)F)n2)cc1', 'O=C([C@H](Cl)Br)c1ccc(Cl)cc1', 'O=C(/N=C\\NC(=S)Nc1cccc2c1CCCC2)Nc1cc(Cl)ccc1Cl', 'Clc1ccccc1Oc1cc(NC[C@H]2CCCO2)ncn1', 'ClCc1ncn2cccc(-c3ccncc3)c12', 'OC[C@@H](NCc1cc(Br)cs1)c1ccc(Cl)cc1', 'Clc1ccc(-c2n[nH]cc2NCc2csc3ccccc23)cc1Br', 'Clc1ccccc1N1NC(c2ccccc2)=NC1', 'Nc1[nH]nc2ncn(Cc3ccccc3Cl)c12', 'Clc1cc(/N=C/c2noc(-c3ccccc3)n2)cc(Cl)c1', 'O=C(/C=C/O[C@@H](O)c1ccc(F)cc1)NCc1ccccc1', 'O=[N+]([O-])c1c(OCc2nccn2Cc2ccccc2)oc2cc(Cl)cc(Cl)c12', 'O=C([C@H](O)c1ccc(Cl)c(Cl)c1)N1CCN(C(=O)c2cccc(F)c2)CC1', 'O=C(Cc1nn[nH]n1)c1ccc2c(c1)Cc1ccccc1C2', 'Oc1ccccc1/C([O-])=N/S(=O)(=O)c1ccc(Br)cc1', 'O[C@@H](c1cc(Cl)cc(Cl)c1Cl)c1ccccc1C1CC1', 'Clc1ccc(-c2cccnc2Cl)c(-n2ccnc2)n1', 'Clc1ccc(Cc2noc([C@H]3COc4ccccc4O3)n2)cc1', 'Oc1ccc(O[C@H](Oc2ccccc2)N2CCN(c3ccccc3)CC2)c(Cl)c1', 'Oc1ccc([N+](=O)c2ccccc2)cc1OSc1ccccc1', 'Oc1cc(O)c(C(=O)Nc2ccc3[nH]nnc3c2)cc1Cl', 'O=C(c1cc([N+](=O)[O-])ccc1[N+](=O)[O-])N1CCC=CC1', 'Oc1cc(Cn2ncc(-c3cc([N+](=O)[O-])ccc3F)n2)ccc1Cl', 'Clc1ccccc1[C@H](CCl)c1c(F)cccc1F', 'Clc1cc(CNC[C@@H](c2ccc3c(c2)OCO3)c2ccccc2)ncn1', 'Oc1ccccc1[C@H](C[NH2+][C@@H](CO)c1ccc(Cl)cc1)c1ccccc1', 'Cc1snc(-c2ccco2)c1CNC(=O)/C=C/c1cccc(OC)c1', 'COc1nn(C)c(CC)c1CNCCNC(=O)c1ncccc1OC(C)C', 'Clc1ccc(-c2nonc2N2CCc3sccc3C2)cc1', 'COc1ncc([N+](=O)[O-])c(/C=N/NC(=O)c2ccc(C)cc2OCC)c1C', 'Oc1cc(-n2nnnc2SCC(=O)c2ccccc2)coc1=O', 'OCc1nnsc1N/N=C/c1cccc(OC2CCCC2)c1', 'Oc1cccc(N2CC[C@H](NCc3ccc(F)cc3F)C2)c1', 'ClCc1cccc(CO)c1I', 'O=[N+]([O-])c1c([O-])c2ncn(CCCc3ccc(F)cc3)c2nc1N', 'O=[N+]([O-])c1c(-c2cc([N+](=O)[O-])ccc2)cc[nH]1', 'Clc1cc(COC2CC2)on1', 'Clc1cc(Cl)c2sccc2n1', 'CN[C@H](CO)c1cccc([N+](=O)[O-])c1', 'Clc1nnnc(-c2cn(Cc3ccccc3)nn2)n1', 'OCc1n[nH]cc1Oc1ccc(Br)cc1', 'Clc1ccc(CNC[C@H]2CCCN(C(=O)[C@@H]3CCCO3)C2)cc1', 'Oc1ccccc1[C@H](NCC1(CO)COC1)c1ccccc1', 'N[C@@H](C1CCC(c2cccs2)CC1)N1CCN(c2ccccc2F)CC1', 'Oc1cc(Cn2c[nH+]c(-c3ccccc3)c2NCc2ccco2)ccc1F', 'Oc1cccc(SC2(C(=O)c3ccccc3)CCOCC2)c1', 'ClCc1ccn(C(=O)c2ccc(F)cc2Cl)c1', 'Oc1cc(F)ccc1C(F)(F)C(F)(F)F', 'Clc1ccccc1C1CC[NH+](Cc2cccc(Cl)c2Cl)CC1', 'ClC(F)(F)OCCCNc1cc(Cl)ccn1', 'Clc1ccc(Br)cc1NCC[S@@](=O)Cc1ccccc1', 'Clc1ccccc1Oc1cnc2cc(F)c(F)cc2c1[O-]', 'Clc1ccccc1Oc1cnc2cc(C(F)(F)F)c[nH]c12', 'Cc1sncc1C(=O)Nc1nc(-c2ncc(-c3ccc(F)cc3)nc2)no1', 'Clc1ccc(Br)cc1-n1ncn(CN2CCOCC2)c1=O', 'OCc1ncnn1C1CC1', 'Clc1ccccc1[C@H](C#N)C(=O)Cc1cccnc1', 'O=C(/C=C\\NCc1csc([N+](=O)[O-])c1)c1ccccc1', 'Oc1ccc([N+](=O)c2cc(Cc3ccc(F)cc3)cs2)cc1', 'C=C[C@@H](OC(=O)Cc1cccc(Cl)c1)c1ccccc1', 'N#Cc1c([N+](=O)[O-])c2cc([N+](=O)[O-])ccc2n1Cc1ccc(F)cc1', 'Clc1ccccc1Nc1nn[n-]c1COc1ccccc1', 'Oc1cc(Br)c2c(c1)nc(C[C@@H](CO)O)n2CC1CCN(c2ccccc2)CC1', 'O=C(/N=C/c1ccccc1F)N(Cc1ccccc1Cl)c1cccs1', 'Clc1ccccc1OC1(N2CCOC[C@@H]2c2ccccc2)CC1', 'Oc1ccc([N+](=O)c2cc(Br)ccc2O)cc1', 'Clc1cc(Br)c(Cc2ccccc2)c2c1COC2', 'Nc1[nH]ncc1NS(=O)(=O)CC1CCCC1', 'O=[N+]([O-])c1cc([C@H]2SCCCN2C(=O)COc2ccc(Cl)cc2)ccc1O', 'ClCc1csc(N[C@H](CO)CO)n1', 'O=C(c1c[nH]nc1-c1ccccc1)N1CCSc2ccccc21', 'O=[N+]([O-])c1c(Cl)ncc(Cl)c1Cl', 'Clc1ccsc1-c1noc(C(=O)Oc2ccc(F)cc2)n1', 'Clc1cc(N2CCN(S(=O)(=O)c3ccc(Cl)cc3Cl)CC2)ccc1[O-]', 'Nc1nc2sc3ccccc3n2c1NCc1cccnc1', 'NS(=O)(=O)Nc1cc(Cl)ccc1C(=O)N(CCn1cccn1)C1CC1', 'Clc1ccc(C(CCn2cccn2)C(F)(F)F)cc1', 'OCc1nc(Cl)cc(C2CC2)n1', 'NS(=O)(=O)Cc1cncc([N+](=O)[O-])c1', 'Clc1cccc2ncn(C[C@@H]3CCCCO3)c12', 'Clc1nnnn1-c1c(F)cccc1F', 'O=C(c1cn[nH]c1)N(C[C@@H]1CCCO1)c1ccccc1', 'Oc1ccc(/N=c2/nc(-c3ccccc3)[nH]n3ncnc23)cc1O', 'Clc1cc(Cl)cn2ncnc12', 'CNS(=O)(=O)c1ccc(Cl)c(C#N)c1Cl', 'Clc1cc(/C=C/N2CCN(c3cccc([N+](=O)[O-])c3)CC2)c[nH]1', 'Clc1ccccc1Oc1cnc2sncc2c1', 'Clc1nncc(S(=O)(=O)N[C@@H](Cc2ccccc2)c2ccccc2)c1', 'CCS(=O)(=O)Cc1ccccc1OC[C@@H]1Cc2ccccc2O1', 'Clc1ccc(-c2cc(SC[C@H]3CCCO3)ncn2)cc1', 'O=C(c1c[nH]cn1)OCc1ccc([N+](=O)[O-])c(Cl)c1', 'Oc1ccc(F)c(/N=C/c2ccn(-c3ccccc3)n2)c1', 'Clc1cccc2[nH]nc(NC[C@H](O)c3ccccc3)c12', 'Clc1cccc2[nH]nc(NC[C@@H](O)c3ccccc3)c12', 'Clc1nnnc(-n2cc(N3CCOCC3)nc2-c2ccccc2F)c1C1CC1', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n\n', 'Clc1cccc2[nH]nnc12\n\n\n', 'Clc1cccc2[nH]nnc12\n\n', 'Clc1cccc2[nH]nnc12\n', 'Clc1cccc2[nH]nnc12', 'Clc1cccc2[nH]nnc12', 'Clc1cccc2[nH]nnc12', 'Clc1cccc2[nH]nnc12', 'Clc1cccc2[nH]nnc12', 'Clc1nnn(CC2CCCCC2)n1', 'COC[C@H](O)C[C@@H](C)C(=O)N1CCN(c2ccc(F)cc2)CC1', 'Clc1ccc(Br)c(SCc2nnc3n2CCC3)c1', 'Clc1nnnn1Cc1csc2ccccc12', 'ClCc1nncn1CC(=O)N[C@H]1CCN([C@@H](c2ccccc2)c2ccccc2)C1', 'O=[N+]([O-])c1c(Br)c(I)c2c(n1)c1cc(-c3ccccc3)ccc12', 'Oc1ccccc1-c1nc(-c2csc(NC3CCC3)c2)no1', 'Clc1ccccc1N1N(CCn2cc[nH+]c2-c2ccco2)CC1', 'Clc1ccccc1-c1csc(Sc2cn[nH]c2-c2cc(Cl)ccc2F)n1', 'O=C(/C(c1cccs1)CC1(C)CC1)Nc1ccccc1Br', 'O=C(CNS(=S)(=O)c1ccccc1)Nc1ccc(F)c(Cl)c1', 'Clc1ccccc1NS(=O)(=O)N[C@]1(CO)CCC[C@@H]1Cc1ccccc1', 'O=C(CCS(=O)(=O)c1cccc(C(F)(F)F)c1)NC[C@H](O)c1ccccc1', 'Clc1ccc(-c2csc3sccc23)cc1F', 'O=[N+]([O-])c1c(F)cc(Cl)cc1OCC(F)(F)F', 'Cc1ncsc1CNC(=O)CCc1nc2c(Br)cccc2[nH]1', 'Oc1cccc(O[C@H]2CCCCO2)c1', 'Clc1cc(F)c(/N=C\\c2ccc(Cl)cc2C)cc1O', 'C=C(C)C[C@@H](NC(=O)CCc1ccc(F)cc1)C1CC1', 'ClCc1cccc(Sc2nncn2C2CC2)c1', 'Clc1ccccc1Oc1cc(SC(F)F)nc2cccnc12', 'Clc1cccc(-c2n[nH]c(CSc3nncc4ccc(Br)cc34)n2)c1', 'Clc1cc(N2CCC[C@H]2c2nnc(-c3ccccc3)o2)ccc1Cl', 'Clc1ccc(-c2nnnn2[C@@H](O)c2ccccc2Br)cc1', 'C[C@@H](NS(=O)(=O)c1ccc(F)c(F)c1)c1nc(-c2ccccc2)no1', 'Clc1cc(F)c(C(=O)N2CCc3nnccc32)cc1F', 'CNC(=O)[C@@H](NC(=O)c1cnn(-c2ccccc2)c1)c1nc2ccccc2cc1Cl', 'Clc1cc(Cl)nc(-n2ccnc2)c1', 'NS(=O)(=O)N(C[C@H]1CCCN1S(=O)(=O)c1ccccc1)c1ccccc1', 'CNC(=O)N(Cc1ccccc1OC)[C@@H](C)c1nccn1C', 'Clc1cc(Cl)nc(SCc2noc(C3CC3)n2)n1', 'O=C(/C(c1cccs1)S(=O)(=O)c1ccc(Cl)cc1)c1ccc(F)cc1', 'O=C(COc1cc(Cl)ccc1N1CCC(=O)[C@@H]1c1ccccn1)N1CCOCC1', 'Clc1ccccc1OC1([C@@H](O)c2ccc(F)cc2)CC1', 'Oc1cc(-n2nnc(/C=C/c3ccccc3)c2N2CCCCC2)cc(Br)c1F', 'O=[N+]([O-])c1ccncc1[C@@H]1CCCN(c2nccc(-c3ccc(F)cc3)n2)C1', 'O=[N+]([O-])c1c(-c2cc(I)cc(Cl)c2)ncnc1N1CCc2ccccc21', 'O[C@@H]([C@@H](O)c1ccccc1)c1ccc2c(c1)CCC2', 'Oc1ccc([N+](=O)Cc2nnnn2-c2ccccc2)c(Cl)c1', 'ClCc1ccn(CCn2cccn2)c1', 'O=C(/C=C1CCSc2ccccc21)c1cc(F)ccc1F', 'Clc1ccc(Br)cc1/N=C\\NC1CCN(c2ccc(F)c(F)c2)CC1', 'Oc1ccc(/N=c2/[nH]ccnc2COc2ccccc2)cc1Cl', 'Clc1ccc(Br)cc1-c1ccsn1', 'Clc1cc(Br)cc(/C=N/N2CCOCC2)c1', 'NS(=O)(=O)N(CCc1ccccc1)c1nc2ccc(Br)cn2n1', 'Clc1ccccc1Nc1nnc(-c2cc(Br)ccc2O)[nH]1', 'Oc1ccc(/C=C2\\SC(=S)N(CC(=O)NCc3ccc(F)cc3)N2Cc2ccccc2)cc1', 'O=C(/N=c1\\onc2n1CCC2)N1CCN(c2ccccc2)CC1', 'Clc1ccc(-c2cc(Cl)n[nH]2)cc1CO', 'ClCc1ccn(C[C@@](O)(c2ccccc2)N2CCOCC2)c1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1Sc1ccccc1F', 'Clc1cc(NSc2nc3ccccc3s2)ccc1Br', 'O=C(COCCC(=O)[O-])NCC1(c2ccc(F)cc2)CC1', 'Clc1ccc(Br)cc1C[C@@H]1CCCN(C(=O)c2cc([N+](=O)[O-])ccc2F)C1', 'NS(=O)(=O)Cc1noc(S(=O)(=O)N2CCN(c3ccccc3Cl)CC2)n1', 'ClCc1n[nH]c2ccc(C(F)(F)F)cc12', 'Clc1ccccc1-c1noc([C@H]2CCCN(C(=O)c3ccccc3)C2)c1Cl', 'O=C(C[NH+]1CCN(C(=O)c2c(F)cccc2F)CC1)N1CCCC1', 'Clc1cc(F)c(OCC(F)F)cc1I', 'CNS(=O)(=O)[C@@H](NCc1cccc(OC)c1)c1ccccc1', 'CNS(=O)(=O)C[C@H]1CCCN1C(=O)CCc1ccccc1OC', 'Clc1nnnn1c1c(O)cccc1N1CCCC1', 'Clc1cccc2[nH]nc(Sc3cccnc3)c12', 'Clc1nnnn1C1CCN(C(=O)c2cc(Cl)c3c(c2)OCCO3)CC1', 'O=C([C@H]1c2ccc(Br)cc2C1)N1CCC(NCc2cccc(F)c2)CC1', 'Clc1ccc(-c2cc[nH]n2)cc1Cn1ccccc1=O', 'O=[N+]([O-])c1c(-c2nc(C3CC3)no2)cnn1Cc1ccc(F)cc1', 'Clc1ccc(Br)cc1C(F)(F)C1CC1', 'Oc1cc(-n2cnnn2)ccc1Nc1ncc([N+](=O)[O-])cc1Br', 'O=C(c1cc(Cl)cn1-c1nccc(C(F)(F)F)c1-c1ccccc1)c1cccc(C(F)(F)F)c1', 'Oc1ccc(O[C@H](C2CC2)c2ccc(F)cc2F)cc1', 'Clc1cccc(/N=N\\Nc2ncccn2)c1', 'Clc1ccc(-c2nn(-c3ccc[nH]c3=O)cc2F)cc1Cl', 'OCc1nnsc1COc1cc(Cl)ccc1Cl', 'Clc1cccc([C@@H](Nc2nccc(C(N)=O)c2)C2CC2)c1', 'Clc1ccc(CO)c(N2C(=O)CNC2=O)c1', 'COc1ncncc1CCCNC(=O)c1cc(-c2ccco2)on1', 'ClCc1csc(Sc2cc(F)ccc2Br)n1', 'O=C(Cc1nonc1C(F)(F)F)C1CCN(S(=O)(=O)c2ccccc2)CC1', 'CN[C@H](CSC)c1ccncc1', 'Clc1ccc(-c2cc[nH]n2)c(F)c1', 'Clc1ccc(-c2ccoc2C2CC(CO)C2)cn1', 'Oc1cc(Br)c(Br)cc1OC(F)F', 'O=C(c1cnn(-c2ccccc2)c1)N1CC2(CCCCC2)Oc2ccccc21', 'O=C(Cc1cn(CC2CCC2)c(=O)[nH]1)Nc1cn[nH]c1', 'O=C(c1cccc(I)c1)[C@H]1COc2ccccc2C1', 'O=[N+]([O-])c1ccncc1N[C@H]1CCN(c2ccccc2)C1=O', 'OCc1n[nH]c(=S)n1/N=C/c1ccc2[nH]cc(-c3ccc(Cl)cc3)c2c1', 'Clc1cc(Br)c(Br)cc1NC[C@@H]1COc2ccccc2O1', 'Clc1ccccc1[C@H](Nc1cccc(F)c1)C(F)(F)F', 'O=[N+]([O-])c1c(Br)cc(Cl)cc1C(=O)N1CCO[C@@H](c2ccccc2)C1', 'O=C(CN(CCn1cccn1)c1ccccc1)NC[C@@H]1CCOC1', 'Oc1ccc(Oc2ccc([C@H]3CCCN(c4ncccn4)C3)nn2)cc1', 'OCS[C@@H](C1CCCCC1)C(c1ccccc1)c1ccccc1', 'Clc1ccccc1-c1cscc1-n1c([C@H]2CCCO2)cc2ccccc21', 'O=C(/N=C/C(=O)Nc1cc(C(F)(F)F)ccc1Cl)c1c[nH]c2ccccc12', 'Clc1cc(NSc2nc3ccccc3[nH]c2=O)n(-c2ccccc2)n1', 'Oc1cc(O)c2nc(NCc3cccnc3)[nH]c(=O)c2c1', 'Clc1nnnc(Oc2c(Cl)cccc2OCC2CC2)c1Cl', 'O=C([O-])/C(CSc1ccccc1)C(=O)Nc1ccccc1', 'O=C([C@H](Cl)c1ccccc1)c1ccc([N+](=O)[O-])cc1F', 'O=[N+]([O-])c1c([N+](=O)[O-])cccc1OC[C@@H]1CCCO1', 'Oc1ccc(F)cc1N(CCCC#N)C1CC1', 'Clc1cc(F)cc2nc(CC3CCN(c4ccccc4F)CC3)nn12', 'Oc1cccc(C[NH2+]Cc2ccc(Oc3ccccc3)cc2)c1', 'OCc1n[nH]cc1-c1cccc(Cl)c1', 'O=C(/C(c1cccc(Cl)c1)N1CCOCC1)c1ccc(C(F)(F)F)cc1', 'ClCc1csc(COc2cccc(Br)n2)n1', 'Clc1ccc(Br)c(-n2nnnc2Sc2ccccc2)n1', 'O=C(c1cn[nH]c1N1CCC[C@@H]1c1ccccc1)N1CCC[C@@H]1c1nc2ccccc2[nH]1', 'Oc1ccc(F)c(O[C@H]2CCN(Cc3ccccc3)C2=O)c1', 'Clc1ccc(Br)cc1-n1cccc1C(=O)N1CCCC[C@H]1CO', 'Nc1c(CSCc2ccccn2)cnn1C', 'Clc1ccccc1Cn1nc2cccnc2c1CSCCO', 'Clc1nnnn1c1cc(Cl)c(Cl)cc1Cl', 'C=CCCOc1ccc(N2C(=O)/C(=C/c3cc(C)on3)C2=O)cc1', 'Clc1ccc(Br)cc1O[C@H](c1ccccc1)[C@@H]1CCCO1', 'O=[N+]([O-])c1cc(N2CCOCC2)cs1', 'O=C(Cc1c(O)cccc1O)N[C@@H]1CCCC[C@H]1c1ccccc1', 'Nc1c(F)cc(Cl)cc1CN1CC[C@@](Cc2ccccc2)C1', 'Oc1ccc(Oc2nc(Br)c3sccc3n2)cc1', 'Clc1ccc(Br)cc1-n1cn[nH]c1=O', 'O=C(c1cncc(N2C[C@@H](C(=O)NC3CC3)C2)n1)N1CCC[C@H](Cc2ccc[nH]2)C1', 'CNS(=O)(=O)[C@@H](c1ccccc1)c1ccccc1Br', 'NS(=O)(=O)/C(C#N)=C/c1ccc(-c2ccc(Cl)cc2)o1', 'O=C(CNc1nnc(C(F)(F)F)s1)Nc1ccsc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(C(=O)N(CCC(F)(F)F)C2CCOCC2)c2ccccc12', 'O=[N+]([O-])c1c([O-])[nH]c(=O)n1CC(=O)Nc1ccc(Oc2ncccn2)cc1', 'O=[N+]([O-])c1ccn(CN2CC[C@@H](c3ccccc3)C2)c1', 'Oc1cc(O)ccc1O[C@H]1CCCc2ccccc12', 'Clc1ccc(-c2n[nH]c3ccccc23)[nH]1', 'Clc1ccc(-c2cccnc2SC[C@@H]2CCCO2)cc1', 'O=C([C@H](c1ccc2c(c1)OCCCO2)N1CCOCC1)N1CCN(Cc2ccccc2)CC1', 'Clc1ccc(-c2nonc2NC2CCCC2)o1', 'Clc1ccccc1O/N=C(\\[O-])CCCn1nc2ccccn2c1=O', 'O=C(CCN1CCOCC1)N(c1ccccc1F)[C@@H]1CCS(=O)(=O)C1', 'Clc1ccc(-c2nonc2)nc1-c1ccc(Br)cc1', 'Nc1n[nH]nc1CNC(=O)C(=O)NC1CC1', 'Clc1ccc(-c2cc[nH]n2)c(Nc2ccc(F)cc2Br)c1', 'O=C(/C=C/c1ccno1)N[C@@H](c1ccccc1)c1ccccn1', 'Clc1nnn(-c2ncc3nnn(-c4ccccc4)c3c2)c1N', 'Clc1cc(/C=C(/Br)C(F)(F)F)ccc1Cl', 'Nc1[nH]nc(COc2cccc(Br)c2)c1N', 'O=C(c1cncc(-c2ccccc2)n1)N1CC[C@@H](N2CCCC2=O)C1', 'O=[N+]([O-])c1ccn([C@H](CO/N=C\\c2c[nH]c3ccccc23)N2CCOCC2)c1', 'N#C[C@H](NCc1ccc(N2CCOCC2)c(Cl)c1)C1(c2cc(Cl)ccc2Cl)CCCC1', 'Clc1ccccc1OCCO[C@H]1C[C@@H]1C1CCCC1', 'Clc1ccc([C@@H](Nc2nccs2)C2CC2)cc1', 'ClC[C@H](O)COc1cccc(F)c1Cl', 'O=C(c1c[nH]nc1CN1CC[NH+](Cc2ccccc2)CC1)N1Cc2ccccc2C1', 'Oc1ccccc1/N=N/c1ccc(C(F)(F)F)cn1', 'O=C(CNC(=O)C(F)(F)F)N[C@@H](C(=O)c1ccc(Cl)cc1)C1CCCC1', 'Clc1ccc(-c2csc3[nH]c(Nc4cccnc4F)nc23)cc1', 'Clc1ccc(Br)c(F)c1C(=O)NC[C@@H]1CCCc2ccccc21', 'Clc1cc(C(=O)COC[C@H]2COc3ccccc3O2)c(F)cc1F', 'O=C(/C(CNc1ccccc1Cl)c1ccc([N+](=O)[O-])cc1)c1ccccc1F', 'O=C(/N=C/c1cccnc1)c1nc2cc(Cl)ccc2o1', 'Clc1cccc(CC2(CO)OCCO2)c1', 'Clc1nnnc(O[C@@H](C(=O)c2ccccc2)c2ccccc2)c1N', 'Oc1ccccc1O[C@@H](c1ccccc1)c1c(N)cnn1C', 'Clc1cc(CC(=O)N2CCC[C@@H]2c2ccco2)ccc1F', 'Clc1cc(F)cc(C(=O)N2Cc3ccccc3[C@@H](c3ccccc3)C2)c1', 'Clc1nncc(OC2CCCC2)n1', 'O=[N+]([O-])c1c(Br)cc(Br)cc1F', 'Clc1nncc(S(=O)(=O)c2ccc(Br)cc2)n1', 'ClCc1ccc[nH]1', 'ClCc1ccc[nH]1', 'ClCc1ccc[nH]1', 'Oc1ccc([N+](=O)COc2ccccc2Cl)cc1O', 'Oc1cc(COC[C@@H]2COc3ccccc3O2)c(Cl)cc1Cl', 'O=C(/C(c1ccccc1)C1=CCCCC1)[C@@H]1CC=CCC1', 'Clc1ccc(Br)c(F)c1C(F)(F)F', 'Clc1nnn(C(C)(C)Cc2ccc[nH]2)c1-c1ccccc1', 'O=C(COc1cc(NC(=O)NC2(c3cccs3)CCOCC2)on1)N1CCCCC1', 'O=C(c1cc([N+](=O)[O-])ccc1N1CCCCC1)N1CCC[C@H](c2nccs2)C1', 'Clc1nnnn1CCOC1CC1', 'Oc1cc(O[C@@H](c2ccccc2)C(F)(F)F)ccc1Cl', 'N#Cc1n[nH]c(=O)c2cc(Cl)cc(-c3ccc(F)cc3F)c12', 'Clc1cc(/C=C2\\SC(=S)N(C3CC3)C2=O)cs1', 'N#C/C(=C\\c1cccs1)C(=O)Nc1cc(Br)cc([N+](=O)[O-])c1', 'Clc1cc(CCl)c[nH]c1=O', 'Oc1ccccc1[C@@H]1CCCN1c1nc2ccccc2[nH]1', 'Oc1ccc([N+](=O)c2cc(Cl)ccc2Cl)cc1O', 'Clc1ccsc1N/N=C\\c1c[nH]c2ccccc12', 'Clc1ccc(Br)cc1/N=N\\n1ccc2ccccc21', 'Clc1ccc([C@@H](Nc2ccc(Br)cc2)C(F)(F)F)cc1Cl', 'Oc1cc(F)cc(CNCC2(N3CCOCC3)CC2)c1', 'Nc1nonc1C(=O)C[C@H](O)c1ccccc1', 'Nc1[nH]nc(CNc2ccc3c(c2)CCCC3)c1[N+](=O)[O-]', 'OC1(SSCc2cccc3cc(F)ccc23)CCCCC1', 'CNS(=O)(=O)CCNC(=O)NCc1cc(F)c(C)s1', 'O=[N+]([O-])c1c(Br)cc(F)cc1NC[C@H](O)c1ccc(Cl)cc1', 'O=C(c1c[nH]nc1-c1cccs1)N1CCC2(CC1)NCCN2C(=O)C[C@@H]1C(=O)N(Cc2ccccc2)CN1Cc1ccccc1', 'O=C(Cc1cn2ccsc2n1)NNC(=O)c1ccc(Cl)s1', 'Clc1ccc(Br)c(F)c1NCC1(CC2CC2)CCOCC1', 'COc1nc(F)ccc1CC(=O)O[P@](=O)(=O)Cc1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])c(F)cc(Nc2cc(Cl)cc(Cl)c2)c1Nc1ccc([N+](=O)[O-])cc1', 'Oc1ccc([N+](=O)[O-])c(I)c1OC[C@H](O)c1ccc(F)cc1', 'O=C(c1ccsc1)c1nc2nc(NC(=O)/C=C\\c3ccccc3)noc2c1Br', 'O[C@@H]1c2ccccc2C(=O)N1Cc1cccs1', 'Clc1nnnc(NCc2ccc(Cc3cc(F)ccc3F)s2)c1', 'Clc1ccc(-c2n[nH]c([O-])n2)cc1', 'Clc1ccccc1NC1([N+](=O)[O-])CCCCC1', 'Oc1ccc([N+](=O)c2cc(C(=O)NCCc3c[nH]c4ccccc34)ccc2F)cn1', 'Clc1ccc(C2SCCS2)cc1O', 'Clc1cccc2cc(F)[nH]/c(=N/CCCNC(=O)c3ccc(OC(F)F)cc3)c12', 'Oc1cc(C(F)(F)F)ccc1[C@H]1CCCN(c2cc(-c3ccccc3)ncn2)C1', 'OC[C@@H]1c2ccc(F)cc2CCN1C(=O)CCCCn1cccn1', 'Clc1ccc([C@@H](O)c2ccc3c(c2)CCO3)cc1Cl', 'O=[N+]([O-])c1cc(N2CCC(c3cc[nH]c3)CC2)ccc1OCC(F)(F)F', 'Clc1ccc(-c2nnnn2Cl)cc1Cl', 'O=C(c1ccc[nH]1)N1CC[C@@H](c2ccccc2)[C@@H]1C#CCc1ccccc1', 'Clc1ccccc1N1N(CCCc2nnc3cccsc23)CCC1', 'O=C(c1ccc[nH]c1=O)/C(=C/c1ccc(Cl)c([N+](=O)[O-])c1)OCC(F)(F)F', 'N#CCS\\C(=N\\OCCc1cccc(Cl)c1)c1ccccc1', 'C[C@@H](NC(=S)Nc1nccs1)C(=O)NC1CC1', 'Clc1nncn1c1c(F)cc(F)cc1F', 'Clc1cc(COCCO)cs1', 'Clc1cc(Cl)c(Br)c(Cl)c1Cl', 'Cc1nonc1N[C@H](C)C(=O)NCc1ccccc1Br', 'O=C(/C(CO)c1ccc(Cl)cc1)c1ccc(Br)c([N+](=O)[O-])c1', 'Clc1ccccc1-c1nc(C[C@@H]2COc3ccccc3O2)no1', 'ClCc1nonc1NC(=O)Cc1csc(N2CCCC2)n1', 'O=[N+]([O-])c1ccc(S(=O)(=O)NCCCC(=O)N2CCCC[C@H]2Cn2cccn2)cc1', 'Oc1ccc([N+](=O)c2cc(C(F)(F)F)ccc2Br)cc1', 'Clc1ccsc1-c1cc(Br)ccn1', 'O=C(c1c[nH]c([O-])n1)N1CCN(S(=O)(=O)c2ccc3c(c2)CCCC3)CC1', 'O=[N+]([O-])c1cc(Br)c(F)c(/C=N/Nc2cn(C)c3ccccc23)c1', 'COc1ncc(/C=C/C(=O)Nc2ccc(Cl)nn2)c(C)c1CC', 'O=C(CNC(=O)C1CCCC1)N1N=C(c2ccco2)C(=O)[C@@H]1C(c1ccc(F)cc1)c1cccs1', 'ClCc1ccnnc1', 'ClCc1ccnnc1', 'Oc1cc(F)ncc1CN(Cc1ccco1)CC(F)(F)F', 'O=C(CSCc1noc(=O)o1)Nc1ccc(F)cc1Cl', 'Clc1nnnn1Cc1cc(Br)ccc1O', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(Cl)cc1Br', 'Clc1ccc(C(N)=NSc2ccc(F)cc2)c(Br)c1', 'Clc1cccc(S(=O)(=O)N(CCSc2nnc(C3CCCC3)n2C2CC2)C2CC2)c1', 'Nc1c(Cl)cccc1N[C@H]1CCN(Cc2ccccc2Cl)C1=O', 'Clc1ccsc1NC(=S)NN=C1CCc2ccccc21', 'O=C(C1=NS(=O)(=O)c2ccccc21)N1CCN(S(=O)(=O)CC2CCC2)CC1', 'O=C(CNC(=O)C1(c2ccccn2)CCOCC1)Nc1cnn(Cc2cc(F)ccc2)c1', 'COc1nnn(CNC(=O)N2CCC[C@@H](C(N)=O)C2)c1N1CCCC1', 'Clc1ccc([C@@H](Br)c2ccc(Cl)c(F)c2F)cc1', 'Oc1cc(O[C@@H]2Oc3ccccc3OC2)ccc1NCc1ccc(Cl)c(Cl)c1', 'NS(=O)(=O)CCCNC(=O)Nn1cccc1', 'Clc1cccc(NC2[C@@H](N3CCC(c4ccccc4)CC3)C2)c1', 'N#C[C@H](F)c1ccccc1OCc1ccc(Br)cc1', 'Clc1cc(Cl)nnc1Cl', 'Clc1cc(Cl)nnc1Cl', 'N[C@@H](Nc1ccc(F)c(Br)c1)c1ccccc1', 'Clc1ccc(-c2cc([N+](=O)[O-])nc(N3CCOCC3)n2)cn1', 'Oc1cc(Cn2cccc2)ccn1', 'O=C(c1cccc2ncccc12)N1CCC(Cn2cc[nH+]c2-c2cccnc2)CC1', 'O=C(/C(Cc1cccs1)Nc1cccc2ccccc12)c1cccnc1', 'O=[N+]([O-])c1c(-c2ccsc2)ncc2ccc(OC(F)F)cc12', 'Clc1cc(/N=C/C2CCOCC2)ccc1OCCCO', 'Oc1ccc([N+](=O)c2ccccc2CO)cc1O', 'O=C(CNS(=O)(=O)c1ccc(F)cc1)N[C@H]1CCSc2ccccc21', 'Clc1cc(/N=C/c2ccc(-c3ccco3)cc2)cs1', 'Clc1ccccc1Oc1cnc2cc(S(=O)(=O)N3CCN(c4cccc(Br)c4)CC3)ncc2n1', 'Clc1ccccc1OCC[NH2+]Cc1cn(-c2ccccc2)nn1', 'Clc1cc(F)c(F)cc1[C@H](O)c1cccc(Br)c1', 'Clc1ccc(Br)cc1N(C[C@@H]1CCCO1)[C@H]1C[C@@H]1c1cc(Br)ccc1O', 'CN=C/C=C(\\C#N)C(=O)[C@@H](c1ccccc1)c1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(SCc2ccccn2)c1-c1ccncc1Cl', 'Clc1ccccc1Oc1cc(Cl)c(Br)s1', 'Nc1[nH]ncc1-c1nnc([C@H]2CCCN(C(=O)CCc3ccc(Cl)cc3)C2)n1Cc1ccccc1', 'Nc1[nH]nc2nc(COc3ccc(F)c(Cl)c3)[nH]c12', 'Clc1ccc(Br)c(Oc2cccc(OC[C@H]3CCC(=O)N3)c2)c1', 'Oc1cc(-n2cnc(Br)n2)ccc1N1CCOCC1', 'O=C([O-])/C=C/c1ccc(-c2ccccc2[N+](=O)[O-])cc1O', 'NS(=O)(=O)/C(C#N)=C\\c1cccc(Cl)c1', 'Clc1ccc(-c2csc3sccc23)nc1', 'N[C@@H]1c2ccccc2O[C@@H]1C(=O)N/N=C/[C@@H]1CCCC2=Cc3ccccc3N21', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1Sc1ccc(Cl)cc1Cl', 'ClCc1csc(SCc2noc(-c3ccccc3O)n2)n1', 'Clc1ccc(CO)c(SCc2nc3ncccc3o2)c1', 'O=C(CNc1cn(C2CCCCC2)c[nH+]c1=O)Nc1cccc(Cl)c1Cl', 'O=C(C1CCN(Cc2cccc(Cl)c2)CC1)N1CCC2(CC1)OCCO2', 'O=C(c1cccs1)[C@H]1Cc2ccccc2CN1C(=O)c1ccco1', 'O=C(C1CC1)N1CC(c2nc(C(F)(F)F)co2)C1', 'O=[N+]([O-])c1c([N+](=O)[O-])c(Br)ccc1OC(F)F', 'Oc1ccc(/N=c2/[nH]c3ccccc3[nH]2)cc1N1CCCC1', 'O=[N+]([O-])c1c(-c2cccs2)nc(-c2ccco2)n1Cc1cccs1', 'O=[N+]([O-])c1ccn(CCNC(=O)Nc2ncn[nH]2)c1', 'OC1(c2ccccc2)CCN(C(=O)c2csc(=O)[nH]2)CC1', 'NS(=O)(=O)Cc1ccccc1C(=O)N1CCC[C@@H]1C(N)=O', 'Clc1ccc(Br)cc1C(=O)[O-]', 'COc1nnc(SCc2ccc(C(=O)[O-])cc2)n1CCOC(C)C', 'C[C@@H](OCC1CC1)C(=O)N1CC[C@@H](Oc2ccccc2F)C1', 'Clc1ccc([C@@H](NNc2nc3ccccc3s2)C2CC2)cc1', 'NS(=O)(=O)/C(Cc1ccccc1)c1cccc([N+](=O)[O-])c1', 'Cc1sn(-c2ccccc2)c1C(=O)N[C@H](c1cccc(Cl)c1)c1ccccc1', 'CN(c1ccnc(Cl)c1)c1ccc(Cl)cc1OC[C@@H]1Cc2ccccc2O1', 'Oc1cc(F)nc(Sc2ccccc2Br)c1', 'O=C(/C=C1CSCCS1)Nc1ccccc1Cc1c[nH]c2ccccc12', 'O=C(COc1ccccn1)N[C@H](Cc1ccccc1)n1cncn1', 'ClCc1ccc2c(c1)OCC[C@@H]2c1ccc(CN2CCCCC2=O)cc1', 'Clc1ccccc1-c1nnc(SCc2cccc(F)c2)n1C[C@H]1CCCO1', 'Clc1cccc(/C(=C/Nc2ccc(F)c(F)c2)C(=O)[O-])c1', 'Oc1cccc(Br)c1OCBr', 'Cc1sc2sc(-c3ccc(S(=O)(=O)NC(C)C)cc3)nc2c1C', 'C[C@@H](NC(=O)Nc1ccccc1)[C@@H]1CCCN(c2ncccn2)C1', 'Oc1ccc([C@H]2CN(Cc3csc(Cc4cccc(F)c4)n3)CC2)cc1', 'Clc1ccccc1-c1ncc(S(=O)(=O)N2CCC[C@H]2C(N)=O)cn1', 'Clc1ccc(C(CCCO)CBr)cc1', 'NS(=O)(=O)CCNC(=O)[C@@H](C#N)c1ccc2c(c1)CCC2', 'Oc1cccc(S[C@@H](C(=O)N2CCC(c3ccc(C(F)(F)F)cc3)CC2)c2ccccc2)c1', 'O=C([O-])/C(=N\\OCc1cccs1)c1ccc(Cl)cc1', 'Clc1cccc2[nH]nc(SCc3ccsc3)c12', 'Clc1cccc2[nH]nc(SCc3ccsc3)c12', 'O=[N+]([O-])c1cc(Br)c(Cl)cc1[C@@H](O)c1ccccc1', 'O=C([C@H](SCc1ccccc1O)Br)c1ccccc1', 'Clc1ccc([C@@H](Br)Cn2ccnc2)cc1', 'Clc1cccc(S[C@@H]2CCOc3ccc(Cl)cc32)c1', 'Clc1ccc(Br)cc1[C@@H](c1ccccc1)S(=O)(=O)c1cccc([N+](=O)[O-])c1', 'Cc1c([C@H](C)NC(=O)c2ccc3c(c2)OCCO3)cccc1[N+](=O)[O-]', 'ClC(=S)O[C@@H](c1ccc(Cl)cc1)c1ccccc1F', 'COc1nn[nH]c1CC(=O)/C=C/c1cccc([N+](=O)[O-])c1', 'Clc1ccc(CNCCCn2ccc[nH+]2)cc1', 'Clc1nnnn1/N=C/c1ccc(Br)cc1', 'O=C(CN(c1cncnc1)S(=O)(=O)c1ccc(Br)cc1)N1CCCCN1', 'O=C(C[NH+]1CCC[C@H]1c1ccc(Cl)cc1)N1CCC[C@@H]1c1ccccc1', 'O=C(c1ccccn1)N1CCO[C@H](c2ccoc2)C1', 'O=[N+]([O-])c1c([O-])c2c(c1-c1ccccc1)c([O-])c(=O)n2CCc1ccccc1', 'NS(=O)(=O)Cc1nc2cccc(Cl)c2[nH]1', 'Oc1cc(-n2nc(C(F)(F)F)cc2Cn2ccccc2=O)ccc1O', 'Oc1ccccc1-n1cccc1C[NH2+]Cc1cnn(-c2ccc(F)cc2)c1', 'Oc1ccc(/C=C(/CO)C(F)(F)F)cc1', 'ClCc1nnc(Oc2cccc([N+](=O)[O-])c2)n1C1CC1', 'Clc1nncnc1/C=C/C(=O)Oc1ccc(Cl)cc1', 'Clc1ccc(Br)cc1[C@@H](Cl)S(=O)(=O)c1ccccc1', 'Oc1cc(Cn2nccc2C(=O)Sc2nc3ccccc3[nH]2)ccc1F', 'ClCc1nnc(COc2ncccc2F)s1', 'O=C(CNS(=O)(=O)c1ccc(Br)cc1)N1CCC[C@@H](c2nc3ccccc3s2)C1', 'Oc1cc(F)ncc1CNC(=O)CSc1ncnc2c1CCCC2', 'Clc1nnnn1-c1cc(F)ccc1Br', 'O=[N+]([O-])c1cc(Cl)c(N2CCc3ccccc3N2)cc1[N+](=O)[O-]', 'Clc1ncnc(N2CC=C(c3ccc(OC(F)F)cc3F)CC2)c1[N+](=O)[O-]', 'Clc1ccc(-c2cc(-c3cc[nH]n3)ncn2)cc1', 'Oc1cccc(O[C@@H](C(=O)[O-])c2ccc(Oc3ccccc3)cc2)c1', 'Clc1ccc(Br)cc1-n1nccc1CSc1nnnn1C1CC1', 'Clc1ccc(Br)cc1[C@@H]1CCCN1C(=O)c1ccc2c(c1)CCC2', 'Clc1ccccc1NCC(C1CC1)S(=O)(=O)c1ccc(Cl)cc1', 'Clc1cccc2nn[nH]c12', 'Clc1cccc2nn[nH]c12', 'Cc1nonc1CCC(=O)NCc1cc(-c2ccc(C#N)cc2)on1', 'O=C(/N=c1/occ2ccccc2c1Cl)c1ccc2c(c1)OCCO2', 'Clc1cccc([C@@H]2CCCN2C[C@@H](O)COc2cccc(Cl)c2)c1', 'Clc1ccc(Br)cc1Sc1ncccc1OCc1ccoc1', 'Oc1cc(-n2cncn2)ccc1NN1CCOCC1', 'O=C(c1ccsc1)c1cnc(-c2cccc(OC(F)F)c2)nn1', 'O=[N+]([O-])c1ccc(N2CCN(Cc3ccc3c(c3)CCCC3)CC2)nc1', 'Clc1nnnn1CC1CCC1', 'Clc1ccccc1O/N=C/Nc1c(Cl)cccc1[N+](=O)[O-]', 'Oc1cc(-n2cnnn2)ccc1N[C@@H](CO)Cc1ccccc1', 'Clc1ccccc1OC1(OC2CCCC2)CCOCC1', 'Clc1cc(N2CCO[C@H](CO)C2)cc(Cl)c1Cl', 'Oc1ccccc1[C@H](Cl)c1ccc[nH]c1=O', 'Clc1nncc(NCCc2nc(C3CC3)no2)c1O', 'Clc1ccc(Br)c(-n2ncc(C(F)(F)F)c2Cl)c1', 'O=[N+]([O-])c1ccn(CC(=O)N2CCSc3ccccc32)c1', 'Clc1ccc(Br)cc1N(Cc1ccns1)C(=O)c1cccc(Br)c1', 'Oc1cc(F)cc([C@@H]2CSc3ccccc32)c1', 'O=C(c1c[nH]c2cc(S(=O)(=O)N3CCOCC3)ccc12)N1CCC[C@H](O)C1', 'O=[N+]([O-])c1c(F)cc2c(C(F)(F)F)ncnc2c1', 'Clc1nncc(-n2cccc2)c1-c1cc(F)ccc1F', 'N#Cc1cc(Br)cc(NC[C@@H](O)COc2ccc(Br)cc2)c1', 'O=[N+]([O-])c1c(Cl)nn(CNc2ccccc2Br)c1NC(=O)c1cccs1', 'O=C(c1ccsc1)N(CCNS(=O)(=O)c1ccccc1)N1CCOCC1', 'O=[N+]([O-])c1c(Cl)n2ncnc2n1Cc1ccccc1', 'Clc1ccccc1-c1nn2c(Cl)cnc2n1Cc1cc(Cl)ccc1Cl', 'Oc1cccc2ncn(/N=C/c3c(Cl)cccc3F)c12', 'Clc1ccccc1C(=O)[C@@H]1CCCC[C@H]1O', 'O=[N+]([O-])c1cccc2c1[nH]c(=S)n2CCS(C)(=O)=O', 'CNc1ncnc(S[C@@H](C)C(=O)NC2CC2)c1C1CC1', 'Clc1ccc(-c2nnnn2-c2cc(C(F)(F)F)cs2)cc1', 'Clc1ccc([C@@H]([C@@H](Br)c2ccccc2)C(F)(F)Cl)cc1', 'O=C(/C=C1/NCCN1C(=O)Cc1ccc(Cl)cc1)Nc1cn[nH]c1', 'Clc1nncc(CO/N=C/c2ccccc2Cl)n1', 'ClCc1nonc1Sc1ccccc1', 'O=C([C@H](CS)c1cc(F)ccc1F)N1CCC(OC2CC2)CC1', 'Nc1cnn(C2CCN(c3ccc(C#N)cc3)CC2)n1', 'OCSCc1nc2cc(Cl)ccc2nc1Cl', 'O=C(/N=C/c1ccc([N+](=O)[O-])cc1)NCCC(F)(F)F', 'Oc1ccc(/C=N\\C(=O)COCc2ccc(F)cc2Cl)c(F)c1', 'Clc1ccccc1C(=O)N[C@H](C(=O)c1ccco1)[C@H]1CCCO1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn2c(=O)n(-c3ccccc3)nc12', 'Clc1cccc2nc(-c3ccco3)nc([O-])c12', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(O)c(CN2CCCCC2)c1Cl', 'Clc1ccccc1[C@H]1[C@H](CO)Cc2ccccc2C1', 'Clc1ccccc1Oc1cc(F)c(-c2ncoc2C2CC2)nc1', 'O=C(c1ccco1)[C@@H]1CCCN1C(=O)CSc1ccc(F)cc1', 'Clc1cc(F)ccc1-n1ncc(C(F)(F)F)n1', 'Clc1cc(Br)cn1N=C\\c1ccc(Br)cc1', 'Clc1nnnc(N2CCC[C@@H](Cc3ccccc3)C2)n1', 'O=C(/N=c1\\oc2ccccc2[nH]1)c1cccc(C(F)(F)F)c1F', 'Clc1ccc(-c2n[nH]c(C[NH2+]c3ccc(N4CCOCC4)cc3)n2)cc1', 'Clc1ccc(-c2cc([C@H](Cl)Cc3ccc(F)cc3)[nH]n2)cc1', 'O=C(CS(=O)(=O)CC[C@H](O)c1ccccc1)N(Cc1ccc(F)cc1)C1CC1', 'O=C(c1cn2ccsc2n1)N1CC(c2ccccc2)=N1', 'O=C(c1c[nH]nc1-c1ccc2c(c1)COCO2)c1ccccc1Cl', 'OCS/C=C/Nc1cc([N+](=O)[O-])ccc1N1CCCCC1', 'N[C@@H](CC(=O)N1CCCC1)c1nc2cc(F)ccc2o1', 'Clc1cc(/C=N/OCc2ccccc2)co1', 'O=C([C@H](CCSc1ccccc1)c1ccsc1)N1CCCCC1', 'Oc1cc(O)c(Br)cc1OC[C@@H]1COc2ccccc2O1', 'Oc1cc(C=NNCc2ccc(CN3CCOCC3)o2)c(F)cc1Cl', 'Clc1cc(/C=C2\\c3ccccc3C(=O)N2c2cccs2)ccn1', 'Oc1cc(F)nc(S/C=C(c2ccccc2)c2ccc(Cl)cc2)c1', 'Nc1[nH]nc(SCC(=O)Nc2nc3ccccc3[nH]2)c1Cl', 'Clc1ccc(-c2csc3[nH]ccc23)cc1O', 'Clc1ccccc1C1([O-])CC1', 'Clc1ccc(Br)cc1[C@@H]1Cc2ccccc2S1', 'O=C([O-])[C@@H](Cc1ccc(F)cc1)N1CCCCC1', 'Clc1cccc2ccc(N3CCO[C@@H](c4cc(Cl)cc(Cl)c4)C3)nc12', 'O=C(c1c[nH]c([N+](=O)[O-])c1)N1CCO[C@@H](c2ccccc2F)C1', 'Cc1snnc1S(=O)(=O)Cc1ccc([N+](=O)[O-])cc1', 'Clc1cc(/N=C/c2ccccc2)c(Br)o1', 'Clc1cc(CCl)cc(I)c1O', 'COc1nnn(CC(=O)N2CCC[C@@H](O)C2)c1Cl', 'ClCc1cncc(COc2ccc(C(=O)[O-])cc2F)n1', 'O=C(CSCNOc1cccc(C(F)(F)F)c1)c1cccc([N+](=O)[O-])c1', 'Clc1ccc(-c2cccs2)[nH]1', 'Clc1ccc(-c2cccs2)[nH]1', 'Clc1ccc(-c2cccs2)[nH]1', 'ClCc1csc2nc(SCc3ccc(Cl)cc3)nn12', 'Oc1cc(Cn2nc(I)c3ccccc3c2=O)ccc1N1CCC2(CC1)OCCO2', 'Oc1ccc(F)c(-c2cc3[nH]c(=O)n(-c4ccccc4[N+](=O)[O-])c3s2)c1', 'Clc1nnc(Sc2cccc(I)c2)nc1N1CCCC1', 'Clc1ccc(-c2ccccc2CN2CCC[C@](O)(C(F)(F)F)C2)cc1', 'Clc1ccc(-c2nonc2)cc1Cl', 'Oc1cc(OC(=O)c2ccc(Cl)cc2F)ccc1N[C@H](c1ccccn1)C1CCCC1', 'Clc1ccc(Br)cc1-n1nc(C(F)(F)F)nc1N1CCOCC1', 'Clc1cc(F)c(Cl)cc1N[C@@H]1CCOc2ccccc21', 'Clc1ccc(-c2ccccc2[N-]S(=O)(=O)c2cc(Br)ccc2F)cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(Oc2ccc(Cl)cc2Br)c1C(F)(F)F', 'Oc1ccc(Oc2cn(-c3ccccc3)ccc2=O)nc1O', 'Clc1ccc(Br)c(/C=C/C(=O)N2CCC[C@@H]2[C@@H]2CCCC2)c1', 'O=C(c1cccs1)N1CCC[C@@H](C(=O)OCc2ccc(C(F)(F)F)cn2)C1', 'O=C([C@H](CNC(=O)CSCc1ccc(Cl)cc1)N1CCCC1)Nc1ccccc1Cl', 'Clc1ccccc1-c1csc(C[C@@H]2C[C@@H]2c2ccccc2)n1', 'O=[N+]([O-])c1cc(C(=O)N[C@H]2CC(=O)N(S(=O)(=O)c3ccccc3)C2)ccc1Cl', 'Clc1cccc2ccc(S(=O)(=O)NCC(c3ccccc3)c3ccc(Cl)cc3)nc21', 'ClCc1n[nH]cc1-c1cccc(Cl)c1', 'Clc1cccc(S[C@@H]2CCN(c3ccccn3)C2)c1', 'C[C@@H](c1ccc([S@](C)=O)cc1)N(C)C(=O)Nc1ccc(C(N)=O)c(Cl)c1', 'Oc1cc(C(=O)N2CC=C(CCn3ccnn3)CC2)ccc1O', 'COc1ncccc1N(CC)Cn1nc(C)cc1C', 'ClCc1nnsc1-c1cnn(-c2ccccc2)c1', 'C=C[C@@H]1N(C(=O)c2ccccc2)Cc2ccc(OC)cc21', 'Clc1ccc(-c2cc[nH]n2)cn1', 'Clc1nncn1c1c(Cl)ccc(Cl)c1Cl', 'Oc1ccc([N+](=O)Cc2nc3ccccc3[nH]2)c2c1CCCC2', 'CNC(=O)[C@H](C)CN(C)C(=O)Nc1ccc(F)cc1Br', 'Clc1ccccc1Oc1cncc([O-])n1', 'ClCc1nnn(C2CCCCC2)n1', 'O=C([C@H](C(=O)[O-])c1ccc(Cl)cc1)c1ccc(F)cc1', 'O[C@@H]1[C@@H](CC(=O)N2CCN(Cc3ccccc3F)CC2)CC(=O)N1C[C@@H]1CCCCO1', 'Clc1ccc(-c2csc3[nH]cnc32)cc1Cl', 'Clc1ccc(-c2csc3[nH]cnc23)cc1Cl', 'N=Cc1c(OCCSc2ccc(F)cc2)nc2ccccn12', 'CN(c1ccn(C)c1)CNC(=O)CCc1c(O)cccc1F', 'Clc1ccc(-c2csc3s/cnc23)cc1Cl', 'Oc1ccc(F)cc1O[C@H](C(=O)NC[C@H]1CCCO1)c1ccccc1', 'OCS[C@@H](c1ccncc1)c1ccc(Cl)cc1', 'OCc1nc(C(F)(F)F)cs1', 'Clc1cc(/C=C2\\C(=O)N=C(Nc3nc(-c4ccccc4)cs3)C[C@@H]2c2ccc(F)cc2)n[nH]1', 'OC[C@@H](Cc1cc(Cl)cc(Cl)c1)Nc1cc(OCc2ccccc2)ncn1', 'Clc1ccccc1-c1cnccc1N[C@@H]1CCN(c2cccc(Cl)c2)C1=O', 'Nc1nc(SCc2cccnc2)cs1', 'O=C(/N=C/NCC1CCCC1)c1ccc(F)cc1F', 'Oc1ccc(/C=C/C(=O)N2CC=C(c3ccccc3)OC2=O)cc1Cl', 'Oc1ccc(/C=N/O[C@H]2CC(=O)N(c3ccccc3)C2)cc1Cl', 'CN=C/c1cc(-c2ccccc2)nc(S)c1C', 'N[C@@H]1[C@H](C(=O)OCCSc2ccccc2)Cc2ccccc21', 'N=Cc1cc2cc(OC)ccc2c(Cl)c1Cl', 'Oc1ccccc1OSC[C@@H](O)c1ccccc1Cl', 'Clc1ccccc1NS(=O)(=O)c1c[nH]nc1C(F)(F)F', 'Oc1cccc2cc(OC[C@H](O)COc3ccccc3)[nH]c(=O)c12', 'ClCc1ccn(C[C@@H]2CCCCN2S(=O)(=O)c2ccc(F)cc2)c1', 'Clc1ccccc1O/N=C\\C(=C\\Br)c1ccc(Br)cc1', 'OC1(NC(=O)CN2CSc3ccccc32)CCCC1', 'Clc1ccccc1N1N=Cc2ccc(Cl)cc2C1', 'Nc1[nH]ncc1C(=O)Nc1cc2c(cc1Cl)c(=O)[nH]n2C', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn2cccc12', 'COC(C)(C)CN(C)C(=O)c1ccc[nH+]c1NC(=O)CC1CCC1', 'O=C([C@H]1[C@H]2CCCCN2C1=O)N(c1ccc(-n2cnnc2)cc1)Cc1ccccc1', 'Clc1cccc(Br)c1N[C@@H]1CCOC2(C[C@@H](COCc3ccccc3)C2)C1', 'Clc1ccc(Cc2c[nH]nc2[C@H]2CC=CC2)cc1', 'Clc1cc(Cl)ncc1O[C@@H](C(=O)c1ccccc1)c1ccccn1', 'O=C([C@H](C#N)c1ccc(Cl)cc1)N1CCN(c2ccc(Cl)cn2)CC1', 'O[C@@H]([C@@H](O)c1ccccc1)CN1CCCCC1=O', 'Oc1cc(-n2nnc(C(F)(F)F)cc2=O)ccc1N1CCOCC1', 'NS(=O)(=O)/N=C/CCc1ccc2c(c1)OCO2', 'Clc1cc(/C=N/OCN(Cc2ccco2)CC2CC2)ccn1', 'Clc1ccccc1Oc1cc(OC[C@H]2CCCO2)nc(-c2ccco2)n1', 'Oc1cc(-n2nnc(C(F)(F)F)n2)ccc1-c1nc2ccccc2s1', 'Clc1ccc(Br)cc1N(Cc1cccc(O)c1)c1cccc[n+]1[O-]', 'ClCc1ccn(Cn2nnc(-c3cccc(Br)c3)n2)c1', 'Clc1ccc(-c2csc3[nH]ncc32)cc1F', 'Clc1ccc(-c2csc3[nH]ncc23)cc1F', 'Clc1cc(NSc2cccc(Cl)c2Cl)ccc1OC(F)(F)F', 'O=[N+]([O-])c1c(Cl)nn(C2CC2)c1NCc1csc2ccccc12', 'NS(=O)(=O)c1ccc(NC(=O)N2CCC(CS(=O)(=O)c3ccc(F)cc3)C(=O)C2)cc1', 'COc1nnnc(NC(=O)CN2CC(=O)Nc3ccccc32)n1', 'Clc1nncc(C2CC2)n1', 'Oc1cc(Br)cc2cn[nH]c12', 'CN[C@H](C#N)c1cc(F)ccc1SCc1ccc(Cl)cc1', 'Nc1nonc1/N=N/c1ccc(-c2ccccc2Cl)c(Cl)c1', 'Clc1ccccc1N1N=CC[C@@H]1C(=O)Nc1cccc2ccccc12', 'Clc1ccsc1[C@@H](Br)c1ccc(Br)cc1', 'OCc1nccs1', 'Clc1ccccc1O/C=C1/SC(=S)N(CC(=O)Nc2ccccc2)C1', 'NS(=O)(=O)/N=c1/c(=N/C(=O)Cc2ccccc2)c(C2CC2)nn1-c1ccccc1', 'Oc1cc(Br)cc(Cl)c1C(F)(F)F', 'Nc1c(Cl)cc(CNC(=O)c2scnc2C2CC2)cn1', 'Clc1ccc(Br)cc1S(=O)(=O)N[C@@H]1CCSc2ccccc21', 'C=CC#CCCNC(=O)Nc1ccc(-n2ncn(C)c2=O)cc1', 'Clc1cccc(SCC/C=C/CO)n1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1', 'Clc1ccc(C2SCCC2)cc1Cl', 'CN(C(=O)CSc1nnnn1C)c1cc(OC)ccc1OCC=C1CCCCC1', 'Nc1c(F)c(N2CCCC2)c([N+](=O)[O-])cc1O', 'Clc1ccccc1[C@H](Br)C1CCC(O)CC1', 'Oc1ccccc1O[C@@H](N[C@@H](Cc1ccccc1)c1ccccn1)NC1(c2ccccc2)CC1', 'COc1nc2scnc2cc1CNCc1cc(F)c(C)cc1OC', 'Clc1nnc(C2CCCC2)n1N1CCCC1', 'Clc1ccc(Cc2nnc3scnc3n2)cc1', 'Clc1ccc(Br)c([C@@H]2C[C@@H]2c2ccccc2)c1', 'O=[N+]([O-])c1c(Br)scc1Oc1cccc2cccnc12', 'CN[C@H](c1ncc(C(C)(C)C)cn1)c1ccc(Cl)cc1', 'NS(=O)(=O)/N=C/C(=O)Nc1cccc(Br)c1', 'Clc1cccc([S@@]CC2(Cn3cc(Cl)cn3)CC2)c1', 'O=C(/C=C\\c1ccc([S@](=O)C(F)F)cc1)NCCNC(=O)c1ccccc1', 'NS(=O)(=O)N1CCC(CNC(=O)N2CC[C@@H](C(=O)Nc3ccccc3)C2)C1', 'Clc1ccccc1O/N=C/Sc1ccccc1[N+](=O)[O-]', 'N=CC1CCN([C@H](C(=O)N2CCCC2)c2ccccc2)CC1', 'O=[N+]([O-])c1ccc2ncn(CC(=O)N3CCC([C@H]4CCOC4)CC3)c2c1', 'NS(=O)(=O)Cc1nccc(CO)c1C(C)C', 'Clc1ccccc1OC1(Br)CC1', 'Clc1nnnc(C(=O)Oc2ccccc2F)c1N', 'ClCc1cccc([C@@H]2CCCN(CN3CCOCC3)C2)c1', 'Clc1cc(Cl)nc2ccc(C(=O)OC[C@@H]3CCCO3)cc12', 'Clc1ccc(Br)cc1-c1ccn(C[C@H]2CCCO2)n1', 'Clc1ccc(Br)cc1[C@@H]1CCCN(C(=O)Nc2ccc3c(c2)OCO3)C1', 'O=C(CSCc1c(N)cnn1Cc1ccccc1F)N/N=C/c1ccc2c(c1)OCO2', 'O=C([C@H](c1c(F)cccc1Cl)N(Cc1c[nH]c2ccccc12)C(F)(F)F)N1CCCC1', 'Clc1ccc(-c2nnnn2C[C@@H]2CC3(CCOCC3)C2)cc1', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n\n', 'Oc1cccc(F)n1\n\n\n\n', 'Oc1cccc(F)n1\n\n\n', 'Oc1cccc(F)n1\n\n', 'Oc1cccc(F)n1\n', 'Oc1cccc(F)n1', 'Oc1cccc(F)n1', 'Oc1cccc(F)n1', 'Clc1ccc(-c2n[nH]c3ncnn23)cc1Cl', 'O=C(/N=C(\\[O-])[C@H]1CC(c2cccc([N+](=O)[O-])c2[N+](=O)[O-])C1)OCc1ccc(Cl)cc1', 'Clc1nncnc1N[C@@H](c1ccccc1)c1ccccn1', 'Clc1ccc([C@@H](Cl)/C=C/Nc2ccc(Cl)cc2)cc1', 'Oc1cc(-n2cncn2)ccc1CNC1=Cc2ccccc2OC1', 'Clc1cc(Cl)nn1C[C@@H](O)COc1ccccc1', 'Oc1ccc(O[C@H]2C=[NH+]C3=C(C(=O)c4ccc(Cl)cc43)[C@H]2/C=C/OCc2ccccc2)cc1OCC(F)(F)F', 'Clc1nnnn1-c1cc(Cl)cc([N+](=O)[O-])c1', 'Clc1cc(F)c(I)cc1I', 'Clc1ccsc1NS(=O)(=O)c1ccc(C(=O)[O-])cc1', 'O=C(/C(c1cccs1)CN1CCOCC1)C(=O)Nc1cccc(Br)c1', 'C[C@@H](Oc1ccccc1[N+](=O)[O-])c1nc(C(F)(F)F)ccc1Cl', 'Clc1ccsc1Sc1ccccc1OC(F)F', 'O=C(CS(=O)(=O)[N-]C(=O)CCCc1cccc(Cl)c1)Nc1ccccc1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(CN(Cc2ccccn2)c2ccc([N+](=O)[O-])cc2)c1NCC1CC1', 'Clc1nncn1-n1cnc(COCCN2CCOCC2)c1C1CC1', 'O=C([C@H](NS(=O)(=O)c1ccc(Cl)cc1)c1ccccc1)N1CCC[C@H](CO)C1', 'ClCc1cc(Br)c[nH]c1=O', 'O=[N+]([O-])c1ccc2c(c1)N(Cc1cc(C(F)(F)F)cs1)CC2', 'O=C(/C=C/c1c(O)oc2cccnc12)COC1CCCCC1', 'Oc1cc(C=NOc2[nH]nc(Nc3ccccc3)n2)ccc1O', 'O=C(COc1cc([N+](=O)[O-])cc(C(=O)[O-])c1)N/N=C/c1cc(Br)ccc1F', 'Clc1ccccc1O/N=C(/NCc1cncc(C(F)(F)F)c1)N1CCOCC1', 'CC[S@](=O)CC(=O)N1CCN(C(=O)Nc2ccccc2-n2cncn2)CC1', 'Clc1cc(N2CCOCC2)ccc1OCc1cc[nH+]c2ccccc12', 'Oc1ccccc1/C(Cl)Oc1cccc(C(F)(F)F)c1F', 'Clc1ccc(COCc2ccccc2)[nH+]c1', 'Nc1[nH]ncc1CN(c1ccccc1)S(=O)(=O)/N=C/c1ccc(Br)cc1', 'Clc1cccc(CC(=O)N[C@H]2CCCC[C@@H]2CO)c1F', 'Clc1ccccc1Oc1cc(Br)c([O-])c(Br)c1', 'NS(=O)(=O)C[C@H](NC(=O)Cc1csc(Nc2ccc(F)cc2)n1)c1ccccc1', 'Cc1ccsc1/C=C/C(=O)N(Cc1ccccc1)Cc1ncns1', 'Clc1ccc(Br)cc1-n1ncc2c1[C@H]2CCCOc1ccccc1', 'N=Cc1c(F)cccc1OCCOC(F)F', 'COc1nc2ncc(C)cc2s1', 'Clc1ccc(-c2csc3[nH]ncc23)c(F)c1', 'O=C(C1=CCCCN1)N1CCN(c2ccc([N+](=O)[O-])c(NN)n2)CC1', 'O=[N+]([O-])c1c(F)cc(OC[C@H](O)COc2cccc(F)c2)cc1Cl', 'O=C(CSCNc1ncccc1Cl)Nc1nc2ccc([N+](=O)[O-])cc2s1', 'Clc1ccsc1S(=O)(=O)OC1CCC(c2nccn2Cc2ccccc2)CC1', 'Clc1ccc(C(N)=NC2=NS(=O)(=O)c3ccccc32)cc1', 'Oc1cccc(Br)c1C[N+](=O)[O-]', 'O=C([C@H](Nc1ccccc1)C(F)(F)F)Nc1ccccc1C#N', 'Cc1sn(-c2ccccc2)c1COC(=O)c1cnn(C)c1C', 'OCc1ccc(N2CCC[C@H](c3nn(CC(F)F)c4ccccc34)C2)c(Cl)c1', 'NS(=O)(=O)CC[C@@H](NC1CC1)c1ccccc1F', 'O=C([O-])/C(=C\\C(=O)Nc1ccccc1OC(F)(F)F)Nc1nc(C2CC2)n(-c2ccccc2)n1', 'NS(=O)(=O)N[C@@H]1CCN(S(=O)(=O)N(C)Cc2cccc(F)c2)C1', 'O=C(/N=C\\c1cc(Cl)ccc1N1CCOCC1)[C@H]1CC(=O)N(c2ccc(OC(F)(F)F)cc2)C1', 'OCc1nnsc1-c1ccc2c(c1)OCO2', 'O=C(/C=C/c1cccc(Cl)c1)NOCC1(O)CCOCC1', 'Clc1ccc([C@@H](Cl)c2ccc3c(c2)OCCO3)cc1F', 'COc1nn(CCC(=O)N[C@@H](C)C[C@H]2CCCCO2)c2ccccc12', 'Clc1ccc(-c2cc[nH]n2)nc1', 'Clc1ccc(-c2cc[nH]n2)nc1', 'Clc1nncc(F)c1C(=O)N1CCCc2ccc(Br)cc21', 'Clc1ccc(-c2nn(-c3cc(C(F)(F)F)ccc3Br)[nH]c2=O)cc1', 'O=C([O-])[C@@H](Cc1ccccc1)c1cc(Cl)ccc1F', 'Clc1ccc(-c2nnnn2C2(C(=O)[O-])CCCCC2)cc1', 'O=C(Cc1c[nH]c(C(=O)[O-])c1)Nc1cccc([N+](=O)[O-])c1', 'O=[N+]([O-])c1ccc2ncn([C@@H](CCl)c3ccccc3)c2c1', 'O=C(c1c[nH]c2ccccc12)N1CCC[C@H](n2ccnc2)C1', 'Clc1nnnn1-c1cc([N+](=O)[O-])ccc1O', 'NS(=O)(=O)/C(=C\\c1ccc(Cl)cc1)c1[nH]c2ccccc2c1Oc1ccc2c(c1)OCO2', 'Oc1ccccc1[C@H](NNC(=O)c1ccc2nccnc2c1)c1ccccc1CO', 'Clc1nncc(-n2cccc2)c1/C=C/C(=O)N1CCc2ccccc21', 'Clc1ccc(C(N)=[NH2+])cc1', 'Clc1cc(/C=N/NC(=S)NC2CC2)nc2cccnc12', 'O=C(Cc1conn1)N1CCC[C@@H](c2nc3ccccc3n2CC(F)(F)F)C1', 'O=C(c1cn[nH]c1)c1ccnc(N2CCC(OC(F)(F)F)CC2)c1', 'Clc1ccc(-c2nnnn2Cl)cc1F', 'O=C(CNc1c(Cl)cccc1[O-])Nc1ccc(C#CCCO)cc1', 'NS(=O)(=O)N[C@@H](c1ccccc1)c1csc2ccccc12', 'Clc1ccccc1O/N=C(\\[O-])c1c[nH]nc1-c1ccccc1', 'N[C@@H]1c2ccccc2CCC1=O', 'Cc1c(Br)cccc1N[C@H](C)C(=O)NNC(=O)c1cccs1', 'C=CCCC[C@@H](C)OC(=O)c1ccc2c(c1)OCO2', 'Clc1ccc(C2C(C(=O)NCc3cccc(C(F)(F)F)c3)CC2)cc1', 'O=[N+]([O-])c1ccc(O)c(NCc2cnn(C3CCCCC3)n2)n1', 'OCc1nnc(-c2ccc(Cl)cc2F)[nH]1', 'Clc1nnnc(NNc2ccc(C(F)(F)F)cc2)n1', 'Oc1ccc(F)cc1CN1CCN(c2[nH+]ccc3ccccc23)CC1', 'Cc1cc(C(=O)N[C@@H](C)c2ccc(Cl)cc2)c2ncnn2c1', 'O=[N+]([O-])c1c(Cl)n2nnc(-c3ccccc3)c2cc1O', 'O=[N+]([O-])c1c(F)cc2[nH]ccc2c1Cl', 'Clc1ccc(Br)cc1C(F)(F)C(F)F', 'N=Cc1cnc(C2CC2)nc1-c1cccc([N+](=O)[O-])c1', 'Clc1nnn(-c2nc(C3CC3)ns2)c1OCc1ccccc1', 'Clc1cccc(F)c1O/N=C/c1cc(Cl)c(Cl)c(Br)c1', 'O=C(c1c[nH]c(=S)s1)N1CCC(OCc2cccc(F)c2)CC1', 'Clc1ccccc1[C@H]1C(c2ccccc2)=Nc2ncnn21', 'Clc1ccc(-c2cc[nH]n2)nc1Sc1ccccc1', 'O=C(Cc1c(F)cccc1Cl)NN[C@H]1CCS(=O)(=O)C1', 'COc1ncc(SCC#N)c(-c2ccc(Cl)cc2Cl)n1', 'N=C1[C@H](NCSc2ccc(F)cc2)C(=O)N(c2ccccc2)N1CCc1ccc(Cl)cc1', 'O=C(c1cn(-c2ncccc2Cl)o1)N1CCOCC1', 'Clc1cc(/C=N/Nc2nc(-c3ccccc3)n[nH]c2=O)cs1', 'O=C(c1cn2ccsc2c1Cl)c1ccc(Cl)c(Cl)c1', 'Clc1cc(F)c([N+](=O)[O-])cc1OC[C@@H]1CCCO1', 'O=C(Cc1cnc2ccccc2c1)Nn1ncc(Cl)c1C(F)(F)F', 'Clc1nnnn1c1c([N+](=O)[O-])cccc1OCC(F)F', 'Clc1nnc([N+](=O)[O-])c(OC2CCCCC2)n1', 'ClCc1cc(Cl)cc2c1OCOC2', 'Clc1cccc(F)c1N[C@@H]1CCSc2ccccc21', 'Clc1cccc(/N=N/NNNc2ccccc2)c1', 'Clc1cccc(Br)c1N1C[C@@H]2[C@@H](C1)CN(Cc1ccc(Br)cc1)N2', 'Clc1ccccc1[C@H]1C[C@@H](CC(=O)Nc2cccc(C(F)(F)F)c2)N(Cc2ccc(F)cc2)C1', 'Clc1cccc([S@@]C2(COC(=O)Nc3ccc(-n4cccn4)cc3)CCCC2)c1', 'Clc1cc(Br)c2nc(OC3CCCC3)oc2c1', 'Clc1ccc(Br)cc1/N=N\\OCc1ccc(Cl)cc1Cl', 'CNC(=O)c1cc[n+](CCC(=O)N[C@@H](C)CCOC)cc1', 'Clc1cccc(-c2ncco2)c1N[C@H]1CCCc2ccccc21', 'Oc1ccc(F)c(F)c1/N=N/OCc1ccc(F)cc1', 'Clc1ccsc1S(=O)(=O)N1C[C@@H](CO)C1', 'Oc1ccccc1[C@H](Nc1cccnc1OC[C@H]1CCCO1)c1ccc(Cl)cc1', 'O=C(c1cccn2cnnc12)N1CCC[C@H](Oc2ccc(Cl)cc2)C1', 'Clc1ccc(-c2cc[nH]n2)c2ncccc12', 'Clc1ccccc1-c1cncc2c1CN2C(F)(F)F', 'Clc1ccccc1CN(CCCn1cc[nH+]c1)C[C@H]1CCCO1', 'CNS(=O)(=O)[C@@H]1CCN(C(=O)Nc2ccc(-c3ncccn3)cc2)C1', 'Clc1ccc(Br)cc1COC(=O)[C@H]1C[C@H]1c1ccc(Cl)cc1', 'Clc1ccc(CO)c(/C(Cl)=[N+]([O-])/C=C/c2ccccc2)c1', 'Oc1cc(O)c(-c2ccccc2)nc1OC[C@H]1CCCO1', 'NS(=O)(=O)/C=C(\\Cc1ccccc1)I', 'Oc1cc(O)cc2[nH]cnc12', 'Oc1cc(O)c2nc[nH]c2c1', 'Clc1ccccc1-c1cscc1-n1nccc1SCc1ccccc1F', 'Clc1nnc(NC(=O)c2ccc3c(c2)OCO3)[nH]1', 'Clc1cccc([S@@](=O)Cc2cccc(I)c2)c1', 'O=C([O-])c1[nH]ncc1C(=O)N1CCN(c2ccc(C(=O)[O-])cc2)CC1', 'O=C(c1ccco1)[C@H]1CC(c2ccccc2)=NO1', 'Oc1cc(-n2nnnc2N2CCN(c3ccc(Cl)c(Cl)c3)CC2)ncn1', 'Clc1nnnn1/N=C/c1ccc([N+](=O)[O-])cc1', 'O=C([C@H]1Cc2cc(Cl)ccc2O1)N1CCC(O)CC1', 'Oc1ccc([N+](=O)c2cc(Br)ccc2F)cc1O', 'N#CCSCCCNC(=O)c1cccn1C(F)F', 'Oc1ccccc1[C@@H](O)Cn1cc(Br)cn1', 'Oc1ccc(F)c(Cl)c1Sc1ccc(Br)cc1O', 'Clc1ccsc1/N=N/CCOc1ccccc1Cl', 'O=C([C@H](Cl)C1CC1)N1c2ccccc2C[C@@H]1CCc1ccc(C(F)(F)F)cc1', 'Clc1nnnc(SCc2cc(F)ccc2F)n1', 'O[C@@H]1c2ccccc2CC[C@@H]1O', 'ClCc1n[nH]c2c1CCC2', 'OCc1nn(-c2ccc(F)cc2)c2c1C[C@@H](NCc1ccco1)C2', 'CNC(=O)C(=O)NC[C@@](C)(O)c1ccc(F)cc1', 'Clc1nncn1CCNC(=O)Nc1nnc(-c2cccc3cccnc32)s1', 'O=[N+]([O-])c1ccncc1Sc1ccc(-n2cccn2)nc1', 'O=[N+]([O-])c1ccnc2ccc(C(=O)N3CCC[C@@H](O)C3)cc12', 'Oc1ccc([N+](=O)C2(CCO)Oc3ccccc32)c2ccccc12', 'Oc1cc(O)ccc1OCCS(=O)(=O)N[C@@H]1CCc2ccccc21', 'Clc1ccccc1Cc1n[nH]c([C@H]2CCc3ccccc32)n1', 'CCCCCNC(=O)c1cc(NC(=O)C=C2CCCCC2)co1', 'O=C(CSC(=O)C1CCCC1)NC[C@@H](c1ccc(Br)cc1)N(CC1CC1)S(=O)(=O)c1ccccc1', 'COc1nn(C)cc1CN[C@@H](C)c1ncc(-c2ccccc2)n1C', 'O=[N+]([O-])c1cc(Br)c(NC2CC2)cn1', 'Oc1ccc([N+](=O)[O-])c(Cl)n1', 'Oc1ccc([N+](=O)[O-])c(Cl)n1', 'Oc1ccc([N+](=O)c2ccccc2Cl)c(I)c1', 'N#Cc1coc(S(=O)(=O)Nc2c(-c3ccccc3)cnn2-c2ccc(F)cc2)c1', 'OCc1nc2ncn(-c3ccc(Br)cc3)c2c(Cl)c1Cl', 'Oc1cccc2cc(/C=N\\C(=O)c3ccc4c(c3)OCCO4)oc12', 'O=[N+]([O-])c1ccn([C@H](CO)c2ccc(Cl)cc2)c1', 'Oc1ccccc1/C(Cl)Cc1ncccc1OC1CCCCC1', 'Oc1ccccc1/N=N\\S(=O)(=O)CCNC(=O)Nc1ccc(I)cc1', 'Clc1ccc(Br)cc1-c1ccs/c(=O)c1C(F)(F)F', 'CNC(=O)COc1c(Cl)cccc1NC(=O)N1CCSC(C)C1', 'ClCc1csc(N2CCNC2=O)n1', 'Clc1cc(Br)c([O-])c(C(=O)N2CCC(c3nccn3CC3CC3)CC2)c1', 'Clc1ccc(-c2nonc2C(F)F)cn1', 'Clc1cc(Cl)nn1/N=C\\c1ccc(Cl)cc1', 'Clc1ccccc1CO[C@H]([NH3+])Cn1ccccc1=O', 'Clc1ccc([C@@H](Cl)C2CC2)c2ncccc12', 'O=C(/C(CO)Cc1ccccc1OC(F)F)CCc1ccsc1', 'Oc1cccc(O[C@@H]2CCOc3ccc(F)cc32)c1', 'Clc1ccc(C(CCCO)C(F)F)cc1', 'CC(C)C[C@@H](C[NH3+])C(=O)Nc1ccncc1', 'ClC(c1cc(C2CC2)nn1-c1ccc(Br)cc1)c1ccc(Cl)c([N+](=O)[O-])c1', 'NS(=O)(=O)C[C@@H](Br)c1ccccc1', 'OC[C@H]1CN(c2nc(-c3ccc(Cl)c(Cl)c3)no2)CCO1', 'O=C([O-])c1ccc(CN[C@H]2CC(=O)N(c3ccccc3)C2)c(F)c1', 'Clc1nncnc1NC1(c2ccccc2)CCOCC1', 'Clc1ccccc1[C@H]1[C@@H](NC(=O)c2ccc3c(c2)CCCC3)CC[NH+]1Cc1ccccc1', 'Clc1cc(Br)c([N+](=O)[O-])cc1Sc1ncccn1', 'Clc1ccc(C(F)(F)F)cc1N1CCCC[C@H]1CN1CCOCC1', 'ClC(c1cc(Br)ccc1F)c1ccc2c(c1)OCO2', 'Clc1nncn1c1c(-c2ccc([N+](=O)[O-])cc2)cc(Br)cc1O', 'C[C@@H](OC(=O)c1ccc(OCC#N)cc1)C(F)(F)F', 'O=C(c1cccs1)N(Cc1ccc[nH]1)c1nc(N2CCOCC2)nc2cccnc12', 'O=C([O-])/C(=N/c1ccc2c(c1)OCO2)N1CCC(Oc2ccccc2)CC1', 'Clc1ccc([C@@H](Cl)C[C@@H]2COc3ccccc3C2)cc1', 'O=C(c1cccs1)[C@H](CO)c1ccncc1', 'Oc1cc(Br)ccc1[C@H](O)c1cc(Cl)ccc1Cl', 'O=C(Cc1coc([C@@H]2CCCO2)c1)Nc1cccc2ccccc12', 'Clc1ccccc1NS[C@H](c1ccccc1)N1CCOCC1', 'Clc1ccccc1NS[C@H](c1ccccc1)N1CCOCC1', 'Clc1nncc(-n2nc(N3CCc4ccccc43)C2)c1C1CC1', 'Clc1ccc(-c2cc([N+](=O)[O-])c([O-])nn2)cc1', 'Oc1ccc(F)c([C@@H]2CCCN2c2cccc(C(F)(F)F)c2)c1', 'O=[N+]([O-])c1c(Cl)nnc2c1CN(Cc1ccccc1Cl)[C@H]2NC(=O)c1ccccn1', 'Nc1c(/C=N\\N(CCCC#Cc2ccccc2)c2ccco2)cc2ccccc2n1', 'Clc1ccc(Br)c(NC2(c3ccccc3)N(Cc3ccccc3)C2)c1', 'O=[N+]([O-])c1cc(F)ccc1N1CCC[C@H]1c1ccco1', 'Oc1cc(-n2nncc2SCc2cn(-c3ccccc3)nc2-c2ccc(F)cc2)ncc1Cl', 'Clc1cccc(C2=NO[C@H](c3cccnc3)C2)c1', 'O=C(CNS(=O)(=O)c1ccccc1C(F)(F)F)N[C@H](c1ccccc1)c1ccc([N+](=O)[O-])cc1', 'O=[N+]([O-])c1cc(OC/C(NC[C@H]2CCCO2)c2ccccc2)ccc1Cl', 'Cc1sn(-c2ccc(F)cc2)c(=O)c1CCC(=O)Nc1ncnc2sccc12', 'Clc1cccc2cc(F)c(SCc3cc(C(F)(F)F)ncn3)cc12', 'CCC[C@@H](C(=O)N(C)[C@H](C)C(C)C)c1ccc(Cl)cc1', 'ClCc1cccc(C2n3c(cc4ccccc4c3=O)C2)c1', 'O=C(c1cnn2c1CCCC2)N1CCO[C@H](c2ccc([N+](=O)[O-])cc2)C1', 'O=C([C@H](c1ccco1)N1CCC(c2[nH]cc[nH+]2)CC1)c1ccccc1', 'ClCc1csc(OC2CC2)n1', 'O=[N+]([O-])c1ccn([C@H](CCO)c2ccc(F)cc2)n1', 'Oc1cc(O[C@@H](C#N)c2ccccc2)ccc1CN1CCOCC1', 'Cc1sc(N(CC[NH3+])Cc2ccccc2)nc1C', 'O=[N+]([O-])c1cc(Cl)n(-c2ccccc2)c1NC[C@@H]1CCCO1', 'Clc1ccc(-c2cc[nH]n2)c([N+](=O)[O-])c1', 'O=[N+]([O-])c1ccnc2cc(C(F)(F)F)cc(-c3cc(Cl)cnc3C3CC3)c12', 'O=C(c1ccsc1)[C@@H]1COc2ccccc2O1', 'Clc1ccccc1OC1(Cl)CCOCC1', 'Oc1ccc(Oc2nc3ccsc3c(=O)o2)cc1Cl', 'Clc1ccc(-c2cccnc2N2CCC[C@@H]2C(F)(F)F)cc1', 'O=C(c1ccc[nH]c1=O)N(CCOc1ccccc1F)[C@@H]1CCS(=O)(=O)C1', 'Nc1nonc1C(=O)N1CCC(c2ncc(C(C)(C)C)nc2)CC1', 'Clc1cc(/N=N/S(=O)(=O)c2ccc(C(F)(F)F)cc2)cc(Cl)c1Cl', 'Nc1nonc1COc1cccc(-c2ccco2)n1', 'Oc1cc(F)ncc1Sc1ncnc2ncnc(Nc3ccccc3)c12', 'O=[N+]([O-])c1ccc(C(=O)N2N=C/C(=C/c3ccccc3Cl)C2=O)cc1', 'Clc1ccccc1CN1CCC[C@@H](c2ccc[nH+]c2)C1', 'Clc1cccc2cc(/C=N/c3cc([N+](=O)[O-])ccc3Cl)[nH]c12', 'Clc1cc(Cl)ccc1-n1cccc1C1CC[NH+](Cc2ccccc2)CC1', 'Clc1ccc(Br)cc1N1/C=C/Nc2ccccc21', 'O=C(C1CCC1)[C@H]1CCCN(c2cc(C(F)(F)F)ccc2Cl)C1', 'O=C(/C=C\\CC1CCCC1)NC[C@@](O)(c1cccs1)C1CC1', 'Clc1nnc(CNc2nccc(OCc3cccs3)n2)o1', 'O=C(/N=c1\\ocnn1Cc1ccccc1)c1cnn(C2CC2)c1', 'O=C(CNCc1c(-c2ccccc2)noc1CO)NCc1ncoc1C1CC1', 'Oc1ccccc1[C@H](O)[C@@H]1CCc2ccccc2C1', 'NS(=O)(=O)Cc1nnc2ccc(Br)cn12', 'Oc1cccc2cc(/C=C/C(=O)N[C@@H]3OCCO3)ccc12', 'Clc1ccc(Br)c(-n2nnc(C(F)(F)F)c2C2CC2)c1', 'NS(=O)(=O)C[C@H](O)c1ccc(Cl)c(Cl)c1', 'CC[C@@H](NC(=O)Cc1cc(C(C)C)no1)C(=O)OC', 'Clc1ccc([C@@H]([O-])N[C@@H](CC(F)(F)F)c2ccccc2)cc1', 'Oc1ccc([N+](=O)CN2CCN(c3ncccn3)CC2)cc1Cl', 'Oc1cc(OCOc2cc(-c3ccccc3)n[nH]c2=O)cs1', 'Oc1ccccc1[C@H]1OCCN1CC[NH+]1CCCCC1', 'Oc1cccc(-c2csnn2)c1O', 'Oc1ccc(/C=N\\NC2=NC=NC3=C2SCC3)cc1', 'Oc1ccc([N+](=O)c2ccc([N+](=O)[O-])cc2)cc1Oc1cccc2c1OCCO2', 'Clc1ccccc1NCc1ccccc1CN1CC[NH+](CC2(CCO)CC2)CC1', 'O=C(C1=NN(c2cccc(O)c2)C(=O)N=C1[O-])N1CCO[C@@H](c2ccccc2)C1', 'Cc1cncc(C(C)C)c1NC(=O)[C@@H](C)Oc1ccccc1C#N', 'CNS(=O)(=O)N1CCN(C(=O)[C@@H](C)Cc2c(C)noc2C)CC1', 'Nc1nc(-c2cccs2)nc2scnc12', 'Oc1ccccc1C1OCOc2cc(S(=O)(=O)NCc3ccccc3)ccc21', 'N[C@@H](CNC(=O)c1cccn2ccnc12)c1cccnc1', 'Clc1ccccc1-c1cn2cc(Br)cnc2c1[O-]', 'O=C(CN(c1cncn(-c2ccccc2F)c1=O)NN1C(=O)COC1=O)c1ccc2c(c1)CCCC2', 'Oc1ccccc1/C([O-])/N=C/c1ccccc1', 'Clc1ccc(Br)cc1OC[C@H]1CC=CCC1', 'O=[N+]([O-])c1c(F)cc(F)cc1NCc1cnn2cnnc2c1', 'COCC(=O)NC1(N(C)Cc2ccc(F)c(F)c2)CCC1', 'C[C@@H](C[NH+]1CCN(CC(=O)N2CCCCC2)CC1)c1ccc(Cl)cc1', 'ClC(c1cc(F)ccc1Cl)c1ccc2c(c1)CCO2', 'OCc1ncc[nH]1', 'CN[C@H](Cn1cncn1)c1ccncc1', 'Clc1cc(Br)cn1N1CC(c2ccc(CO)cc2)C1', 'Clc1cc(Nc2c([O-])n3ccccc3n2)ccc1Br', 'Oc1cc(Br)ccc1Oc1ncc(Br)c(O)c1F', 'Clc1nnn(C[C@@H](CO)Oc2ccccc2)c1CN1CCOCC1', 'NS(=O)(=O)/N=C/c1ccccc1O', 'O[C@@H]1C[C@@](Oc2ccccc2)c2ccccc21', 'O=C(c1cnn(C[C@@H]2CCCO2)c1)N1CCC(c2nc3c(F)cccc3s2)CC1', 'ClC[C@H]1OCCc2ccccc21', 'O[C@@H](c1cc(Br)cnc1Cl)Cc1ccc(Cl)cc1F', 'Oc1ccc(/C=N/NNc2ccc[n+]([O-])c2)cc1', 'O=C(CNS(=S)(=O)c1ccccc1)NC1CC1', 'ClCc1ncnc(C2CC2)n1', 'Clc1nnnn1-c1cc(Br)cc(Br)c1', 'O=[N+]([O-])c1ccncc1O[C@H](Nc1ccc(Br)cc1)C(=O)NC1CCCCC1', 'O=C(/N=C/c1cc(-c2ccc(F)cc2)n(-c2ccccc2Cl)n1)NCC1(O)CCOCC1', 'C[C@@H]([C@@H](O)COc1ccccc1)N(C)c1ccccc1', 'O[C@@H](c1ccc(Br)o1)C1CCN(S(=O)(=O)c2ccc(F)cc2)CC1', 'O=[N+]([O-])c1cc(F)c(OC[C@@H]2CCCCO2)cc1Br', 'Oc1cccc2nn[nH]c12', 'Oc1cccc2nn[nH]c12', 'Clc1ccc(Br)cc1[C@@H]1OCCc2ccccc21', 'Clc1cccc(Br)c1OC[C@H](O)COc1ccc2nc[nH]c2c1', 'O=C(/N=c1c(N2CCN(c3ccccc3)CC2)nc2cc3c(cc12)CCCC3)c1ccco1', 'O=[N+]([O-])c1c(Cl)nsc1-c1cccc(F)c1', 'Clc1ccccc1OC1CCC([NH3+])CC1', 'Nc1cnn(-c2ncccn2)c1Br', 'O=C(c1c[nH]nc1NC[C@H](c1ccco1)N1CCc2ccccc21)N1CCOCC1', 'Oc1ccc(/C=C/NC(=O)CSc2nc3ccccc3nc2N)c(Cl)c1', 'NS(=O)(=O)/C=C\\C(=O)c1ccc([N+](=O)[O-])cc1C(F)(F)F', 'Clc1ccc([C@@H]([O-])Cc2ccc([N+](=O)[O-])cc2)cc1', 'Clc1ccc(-c2cc(N3CCOCC3)[nH]c2Br)nc1', 'Clc1cccc([C@@H](CCc2c[nH]c3ccccc23)N=C2CCOCC2)c1', 'NS(=O)(=O)/N=C\\c1ccccc1Br', 'O=[N+]([O-])c1ccc(S[C@@H](C(=O)N2CCCC2)c2nccs2)cc1', 'Oc1ccc([N+](=O)c2cc([N+](=O)[O-])ccc2O)cc1O', 'Clc1ccc(-c2nonc2)cc1F', 'O=C(CSCc1cccnc1)NN[C@@H]1CCc2ccccc21', 'Clc1ccccc1-c1cscc1C[NH2+]Cc1ccccc1', 'O=C([O-])/C=C/c1cn(-c2ccccc2)nn1', 'Clc1cc(F)c([C@H]2CC(=O)N2c2ccc(Cl)cc2Cl)cc1', 'O=C(/N=C\\N1CCN(c2c(C#N)cc(F)cc2F)CC1)c1ccccc1', 'C=C(C)Cn1c(C)c(C(C)=O)c2c1ncn2C', 'NS(=O)(=O)/N=C(/c1ccccc1)[C@@H]1CCCN(c2ccc(Cl)cc2)C1', 'O=C(CN(c1cnn(C(F)F)c1)S(=O)(=O)c1ccccc1)N1CCc2ccc(Cl)cc2C1', 'ClCc1csc2nc(-c3c[nH]c4ccccc34)oc12', 'Clc1nncn1C1CCN(c2ccc(C(F)(F)F)nc2Cl)CC1', 'O=[N+]([O-])c1c(Cl)nsc1N/N=C\\c1ccc(-c2cccnc2Cl)cc1', 'O=C(C/c1c(F)cccc1F)[C@@H]1Cc2ccccc2S1', 'Clc1nnn(-c2cccc(Cl)c2Cl)n1', 'Clc1ccc(Br)c([C@@H]2CCCN(S(=O)(=O)c3cc(F)cc(F)c3)C2)c1', 'Clc1ccccc1O/C=C[C@@H](O)COC1CCCC1', 'O=[N+]([O-])c1cc([C@H]2OCCN2CCCN2CCOCC2)c2cn[nH]c2c1', 'O=C([O-])CSc1ccc2c(c1)CCC(=O)N2', 'Oc1cc(-n2nncc2Br)c2ccccc2c1O', 'Oc1cc(-n2cc(F)cc2Cn2cc[nH+]c2)ccc1Nc1ccc(F)c(F)c1', 'Oc1ccccc1OCCN(C[C@@H]1CCCO1)c1sccc1C1CC1', 'Clc1ccc(-c2n[nH]c3[nH]cnc23)cc1Cl', 'O=C(c1cc(Br)c[nH]1)N1CCC[C@H](CO)C1', 'Clc1nnn(-c2ncn(-c3cccc(Cl)c3)c2-c2ccn[nH]2)n1', 'CCn1ncnc1C[C@H](NC(=O)c1cnn(-c2ccc(C)cc2)c1)C(C)C', 'Clc1cc(NSc2ncc(-c3ccccc3Cl)n2C2CC2)cc2c1CCCC2', 'Clc1ccccc1OC1([NH+]2CCCCC2)Cc2ccccc2C1', 'Clc1cccc2cc([C@H](O)C(F)(F)F)cc(-c3ccccc3)c12', 'Clc1ccc(Cc2ncoc2CNc2ccc(-n3cccc3)o2)cc1', 'O=C(COc1cc([N+](=O)[O-])ccc1NC(=O)N1CCN(c2cc[nH+]cc2)CC1)c1ccc(Cl)cc1', 'O=C(C1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1)N1CCC(c2nc(-c3cccnc3)c[nH]c2=O)CC1', 'NS(=O)(=O)/N=C(/NCc1cccc(Cl)c1)n1nc(-c2cccs2)c2ccccc21', 'OCc1nc([O-])c(-c2ccccc2)o1', 'O=C(c1ccsc1)[C@@H]1CC(=O)N(Cc2cccc(F)c2)C1', 'Clc1cccc([S@@]c2cnccc2[O-])c1', 'OCc1nnn(CC2CC2)c1Br', 'O[C@@H](c1ccc2c(c1)CCC2)C(F)(F)F', 'Clc1ccccc1NS(=O)(=O)N[C@H]1CCS(=O)(=O)C1', 'Clc1cccc2[nH]c(=O)n([C@@H](CN3CCO[C@H](O)CCCO)C3)c12', 'Oc1cc(Br)c([C@H]2CCCN2CC(=O)NCc2ccco2)cc1F', 'COCc1c(C)sc2occ(-c3ccc(OC(F)F)cc3)c12', 'O=C(c1c[nH]nc1C1CC1)c1cc(-c2ccc(Br)cc2)nc2ncnn12', 'O=C(CNc1nn(Cc2cc(F)cc(Cl)c2)c(=O)c2occc12)Nc1cccnc1Cl', 'Clc1ccc(Br)cc1N1/C=Nc2ccc(Cl)cc2C1', 'O=C(CNc1nn(-c2cccc(C(F)(F)F)c2)c2c1NCCC2)Nc1ccc(F)c(F)c1', 'NS(=O)(=O)/C=C(\\CC#N)C(=O)Nc1ccc2c(c1)OCO2', 'CN=C([C@@H]1COC1)c1ccccc1C(F)(F)F', 'O=C(CS(=O)(=O)Cc1ccccc1)Nc1cc(Oc2ccccc2)c(S(N)=O)cc1', 'CC[C@@H]1c2ccncc2CN1C(=O)c1ccc(NC(=O)C(C)(C)C)cc1', 'ClCc1csc2ncncc12', 'Clc1nnn(-c2ncn[nH]2)c1-c1ccc(Cl)cc1', 'Clc1ccccc1-c1cncc([C@H]2CCOC2)n1', 'Clc1ccsc1NC1=NS(=O)(=O)c2ccccc21', 'Clc1nnn(Cc2ccc(F)cc2Cl)c1NOC(=O)CC1CCCC1', 'Clc1ccc([C@@H](O)[C@@H](Cc2cccc(F)c2)c2cccnc2)cc1', 'Oc1cc(-n2cc(Cn3cncn3)c(Cl)cc2=O)cc(OCC(F)F)c1', 'CNS(=O)(=O)c1c(N=N/Nc2ccc(Cl)cc2C)nc2ccccc2n1', 'Oc1ccc([N+](=O)Cc2noc3ccccc23)c2ncccc12', 'COc1nc(NC[C@H]2CCCO2)ccc1S(=O)(=O)NC1CCC(C)CC1', 'Clc1ccc(-c2cc(F)c(C[NH2+]Cc3cccs3)cc2)cc1', 'Clc1cc(F)c(Cl)cc1NCCc1nc(Cl)ns1', 'Clc1ccc(C[NH2+][C@H](c2ccccc2)c2cccnc2)cc1', 'Clc1ccc(Cc2noc(C[NH+]3CCN(c4ccc(Cl)cc4)CC3)n2)cc1', 'Oc1cccc(CC(=O)N2CCC[C@H]2[C@@H]2CCCC2=O)c1', 'Oc1cc(O)c2scc(Br)c2c1', 'Clc1ccc(Br)cc1[C@@H]1CSCN1c1ccc(F)cc1', 'Clc1nncnc1Nc1ccc([N+](=O)[O-])cn1', 'Oc1cc(-n2cnnn2)c(NC2CC2)c(Cl)c1', 'N#C[C@H](C(=O)[C@@H]1CCCN(C(=O)Cc2ccccc2Cl)C1)c1ccccc1', 'Cc1cc(F)ccc1NC(=O)[C@@H]1CCCN(C(=O)C(=O)NCc2ccc[nH]2)C1', 'Clc1cc(/C=Cc2[nH]nc3c2CCCC3)c(OCC(F)(F)F)cc1', 'Clc1ccc(-c2nnnn2Cl)cc1[N+](=O)[O-]', 'Clc1ccc([C@@H](NNC(=O)c2ccccc2Cl)N2CCCCC2)cc1Cl', 'Clc1cc(/N=C/c2sccc2C(F)(F)F)ccc1Cl', 'Clc1ccccc1Nc1cnc([O-])nc1C1CC1', 'O=C(CSCC1=C2C=CC=C2C[C@H]1Cc1ccccc1)OCc1ccccc1', 'Clc1cc(Br)c([C@H](O)c2ccccn2)cc1F', 'O=C(c1c[nH]c2cc(C(F)(F)F)cnc12)N1CCC(c2cnc[nH]2)CC1', 'Oc1ccc([N+](=O)c2cc(O)ccc2F)cc1O', 'Clc1ccc(-c2nnc(CSC[C@@H]3CCCO3)s2)cc1', 'Clc1nnnn1c1c(F)cc(Oc2ccc(F)cc2)c(Cl)c1O', 'CC[S@](=O)Cc1cc(NC(=O)[C@@H]2CCCN2C(=O)N2CCCC2)cc(F)c1', 'Clc1cccc2cc(OCn3c(CNC(=O)c4cccc(Cl)c4)no3)cnc12', 'OC[C@@H](N[C@@H]1CCN(CC2CC2)C1)c1nc2ccccc2[nH]1', 'C[C@@H](CCc1ccc(Cl)cc1)c1nnc(NC(=O)C(=O)NC2CC2)s1', 'Clc1ccsc1-c1cc2cccc(F)c2nc1NC1COC1', 'ClC[C@@H]1OCCc2c1[nH]c1ccccc21', 'O=C(/C=C/c1c(Cl)cccc1Cl)NC[C@@H]1CN2CCCC[C@H]2CO1', 'Clc1nnnc(-c2ccco2)n1', 'CN[C@H](NC(=O)c1ccccc1Cl)C(=O)NCCc1nccs1', 'Clc1ccc([C@@H](Br)C[C@@H]2CCCO2)cc1', 'Clc1cc(Cl)nn1CCOC1CC1', 'Clc1nncc(S/c2ccc(F)cc2F)n1', 'O=C([O-])[C@@H]1CC(=O)N(CCSc2ccccc2)C1', 'Oc1ccc([C@H](O)[C@@H](NC(=O)NC2CCCC2)c2cccs2)cc1', 'Clc1ccccc1OCCS[C@H]1[C@@H]2CCC[C@@H]2C1', 'Oc1ccc([N+](=O)[O-])c(N[C@H]2CCc3cc(F)ccc32)c1', 'O=[N+]([O-])c1c([O-])cnn1-c1ccccc1', 'Clc1ccsc1-c1cscn1', 'Oc1cc(F)ncc1CNc1cc(C(F)(F)F)c2c(c1)CCCC2', 'OCc1n[nH]c(N2CCOCC2)n1', 'Clc1nnnc(CSc2nncn2-c2ccccc2)n1', 'O=C(c1ccsc1)[C@@H]1Cc2ccccc2C(=O)O1', 'O=C(c1cccnc1)N(CC1CCN(C(=O)c2cnco2)CC1)C[C@H]1CCCO1', 'Clc1cccc([S@@]CCn2ccnc2OCC(F)F)c1', 'Oc1ccc([N+](=O)[O-])c(I)c1O', 'O=C([C@H](Cl)CS)c1cccc(F)c1', 'Clc1ccccc1N1N=C/c2ccccc2[C@H]1c1ccccc1F', 'Oc1ccc(F)cc1OC[C@@H]1Cc2ccccc2S1', 'ClCc1nnc(Sc2ccc(Cl)cc2)[nH]1', 'Clc1ccc(CO)nc1O/N=C/c1c(F)cccc1F', 'Clc1cc(Br)cnc1I', 'Nc1c(Cl)nsc1NCc1csc2ccccc12', 'Clc1ccc(-c2nonc2)nc1Nc1cc(Br)ccc1OC(F)F', 'COc1nc(N(C2CC2)[C@@H](C)C2CC2)ccc1-c1ccco1', 'COc1ncc(-c2ncc[nH]2)cc1C#N', 'O=C(c1cc(Br)c2c(c1)OCCO2)N1CCC[C@@H]1c1cccc(F)c1', 'Clc1ccccc1C(=O)OC[C@@H]1CN2CCC[C@@H]2CO1', 'Oc1ccc(O[C@H](CO)C(F)(F)F)cc1', 'O=C(c1c[nH]cn1)N[C@H](CO)c1ccccc1F', 'O=[N+]([O-])c1cc(OCOCC[C@H]2CCCCO2)ccc1Cl', 'Clc1ccc(Br)cc1[C@@H]([O-])CSc1ccccc1', 'Clc1ccsc1-c1c[nH]c(=O)[nH]c1=O', 'Clc1nnn(-c2ccccc2)c1N[C@H]1CCOc2ccccc21', 'NS(=O)(=O)/C=C(c1cccs1)S(=O)(=O)Cc1ccccc1', 'Clc1cc(N2CCNC[C@@H]2c2cccc3ccccc23)ccn1', 'Clc1cccc([C@@H](Sc2ccccc2)[C@H]2CCCO2)c1', 'Clc1ccc(Br)c(Sc2nccnc2N2CCNC2=O)c1', 'CCN1C(=O)N=C(NNc2ccc(C)cc2C)C1=O', 'Clc1nnn(C2CCCC2)c1-c1ccc(Br)s1', 'Clc1ccc(-c2cc[nH]n2)c(Br)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1Cl', 'N=Cc1ncoc1-c1ccccc1OC1CCN(CC(F)(F)F)CC1', 'Nc1cnnn1-c1ccc(F)cc1C(=O)N[C@H](c1ccccc1)C1CC1', 'O=C(c1cnc(-c2ccccc2)o1)N(Cc1ccccc1F)[C@H]1CCS(=O)(=O)C1', 'N=Cc1nonc1Cc1ccc(Cl)cc1', 'NS(=O)(=O)[C@@H]1CCCN(C(=O)CSc2cccc([N+](=O)[O-])c2)C1', 'Clc1cc(F)cc2nccn12', 'C[C@@H](N[C@@H](CC#N)c1ccccc1)c1ccccc1Br', 'Oc1cccc(O[C@H]2CCN(c3ncccn3)c3ccccc32)c1', 'Clc1ccccc1O/N=C(/c1ccncc1)S(=O)(=O)c1cccc(Cl)c1Cl', 'Clc1ccc(Br)cc1-n1nnc(C(F)(F)F)n1', 'Clc1ccc([C@@H](NN/c2ccc(Cl)cc2Cl)c2cc(F)ccc2F)cc1', 'CN(c1ccccc1)S(=O)(=O)c1ccc([N-]S(=O)(=O)c2cccc(F)c2)cc1', 'Clc1ccccc1Nc1cnc(S(=O)(=O)N(C(C)C)N2CCOCC2)c2ccccc12', 'Clc1ccccc1-c1nn(C2CC2)c(-n2nc([O-])c3ccccc23)n1', 'Clc1ccc(-c2cc([C@H](Nc3cncc(F)c3)Nc3ccc(F)cc3F)[nH]c2)cc1', 'ClC[C@H]1CCCC[C@@H]1c1ccccn1', 'Clc1ccc(Br)cc1-n1nnnc1NC[C@@H]1CCCCO1', 'O=[N+]([O-])c1cc(N2CCC[C@@H]2c2nc3ccccc3s2)cc([N+](=O)[O-])c1', 'O=C(CSCN1C(=O)[C@H]2CCN(Cc3ccccc3)C2=C1CN1CCOCC1)c1ccccc1', 'Clc1cc(/C=N/N2CCCCC2)c2c(c1)OCO2', 'O=C([O-])/C=C(/C(=O)NCc1noc(-c2ccc(F)c(F)c2)n1)c1ccccc1O', 'Oc1ccc(O[C@@H](Cn2cccn2)C(=O)c2c[nH]c3ccccc23)cc1F', 'O=C(/C(c1cccs1)C(=O)c1cccs1)c1ccc(OC(F)(F)F)cc1', 'O=C([O-])c1ccc(-n2cccn2)cc1C(=O)NC[C@H]1Cc2ccccc2O1', 'O=C(/C=C\\c1c(OC2CCCC2)cc(O)c2c1CSCO2)N1CCCCC1', 'Clc1nncn1/c1nc2ccc(Br)cc2o1', 'Oc1ccccc1[C@@H]([O-])CCCNC(=O)[C@H]1COc2ccccc2O1', 'Cc1sc(NC(=O)C(C)(C)C)cc1C(=O)N(C)CCC(F)(F)F', 'O=[N+]([O-])c1c(Cl)nsc1Nc1cccc(Br)c1', 'Oc1cc(-n2nnnc2CN2CCN(c3ccccc3F)CC2)ccc1NC[C@@H]1CCCO1', 'Clc1ccsc1N[C@@H]1CCCN(c2ccccc2F)C1', 'O=C(Cc1non1C1CC1)NCc1ccnc(-n2cccn2)c1', 'Oc1ccccc1-c1c([C@@H](Cl)c2ccccc2)ncnc1Cl', 'O=[N+]([O-])c1c(Cl)nsc1Nc1ccc(Cl)cc1F', 'O=C(c1cnn2cc([N+](=O)[O-])cnc12)N(Cc1ccccn1)S(=O)(=O)c1ccc(F)cc1', 'Oc1ccc(O[C@@H](C2CCCC2)C(F)(F)F)cc1', 'Clc1ccc([C@@H]([O-])NC2CCCC2)cc1', 'Clc1cccc(NC2[C@H](Cc3ccccc3)O2)c1', 'Oc1ccc(/C=C/C[NH2+]Cc2ccccc2O)cc1', 'Clc1ccc(-c2ccccc2[N-]CCO)cc1', 'O[C@@H]1C[C@@]1(O)CCc1ccc(Cl)cc1', 'Oc1cc(O[C@@H](Cc2ccc(Br)s2)C(F)(F)F)c(OC[C@H]2CCCO2)cc1', 'Oc1ccccc1-c1c(-c2ccc3c(c2)OCCO3)n[nH]c1N1CCOC[C@@H]1C1CC1', 'O=C(c1ccsc1Cl)Oc1cccc2c[nH]cc12', 'O=C(/N=C/N1CCN(Cc2ccco2)CC1)C(F)(F)F', 'NS(=O)(=O)NC[C@H](c1ccc(Cl)cc1)N1C(=O)COc2ccccc21', 'Clc1cccc(Br)c1CSCCCCC[NH+]1CCCCC1', 'Clc1ccc(C[NH+](CC(=O)N2CCC[C@@H]2CC2CCCCC2)C2CC2)cc1', 'COc1nc(-n2nc(C)cc2CC(=O)N2CCCc3cc(C)ccc32)cs1', 'Clc1ccccc1N1N=CC(c2ccccc2)c2ccccc21', 'O=C(/C=C/O)C[C@H](c1ccccc1)NC(C)(C)C', 'O=C(Cc1nc(N2CCOCC2)nc2c(C(F)F)ncn12)Nc1ccc([N+](=O)[O-])cc1', 'O=C(/N=C\\c1ccc(Cl)cc1)N1CCc2cc(Cl)cc([N+](=O)[O-])c2C1', 'Oc1ccccc1Oc1cncc([O-])n1', 'Clc1cc(Cl)c(Cl)nc1N[C@@H]1CCOc2ccccc21', 'Oc1ccc(/C=C/N=C([O-])Oc2ccc([N+](=O)[O-])cc2)c(OCc2ccccc2)c1', 'O=[N+]([O-])c1ccncc1NCCC(=O)N1CCC[C@@H](c2nccs2)C1', 'Clc1ccsc1C(F)(F)F', 'N#C[C@H](C(=O)Cc1cccc(Cl)c1)CCS(=O)(=O)c1ccccc1', 'Clc1cc(Br)c([O-])c(=O)n1/N=c1s/c(=C\\c2ccccc2)n1Cc1ccccc1', 'Clc1ccc([C@@H](NN2CCOCC2)c2cc(F)ccc2F)cc1', 'Oc1ccc([N+](=O)[O-])c(-n2ccnc2)c1C(F)(F)F', 'Oc1ccc([N+](=O)[O-])c(Cl)c1Oc1ncc(Cl)c(Cl)c1Cl', 'N[C@@H](N1CCN(c2ccc(F)cc2)CC1)c1cc(F)cc(Br)c1', 'Clc1ccc([C@@H](OC(=O)C2Oc3ccccc3F)C2)cc1', 'O=[N+]([O-])c1c(Cl)n2nnnc2n1Cc1ccc(F)cc1', 'O=[N+]([O-])c1c(Cl)n2nnnc2n1Cc1ccc(F)cc1', 'O=C(c1cn(-c2ccccc2)nn1)O[C@@H](c1cccs1)C1CC1', 'Clc1ccc(SCCCC(=O)N2CC[C@H](c3ccccc3)C2)[nH]1', 'Clc1ccccc1-c1csc(SC[C@H](O)C2CC2)n1', 'Clc1cc(Br)cnc1SCc1ncn[nH]1', 'COc1nnn(CC2CCCC2)c1[O-]', 'O=[N+]([O-])c1c(Br)c(-c2cccc(O)c2)nc2[nH]ccc12', 'O=C(c1c[nH]c(N/N=C/c2cccc(Cl)c2F)n1)OCc1ccncc1', 'Clc1cc(CCl)nnc1-c1cnccn1', 'Oc1ccc(O[C@H](CC(F)F)c2cccc(Cl)c2)c(F)c1', 'Clc1cccc(Br)c1S[C@@H]1CN(C(=O)c2cccc(F)c2)CC1', 'O=C(c1c[nH]cn1)[C@H]1CC(=O)N(Cc2ccc(Cl)cc2)C1', 'Oc1ccc(F)cc1OCCN1CCN(c2cc[nH+]cc2)CC1', 'N[C@@H](CCSc1ccncc1)c1ccc(Cc2ccccc2)s1', 'O=C(Cc1noc(C2CC2)n1)[C@@H]1CCCCN1S(=O)(=O)c1ccccc1', 'Clc1nnn(-c2ncc3ccccc3n2)n1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1[N+](=O)[O-]', 'Clc1ccccc1C(CO)(Cl)[C@H](O)c1ccccc1', 'ClC[C@H](Cc1ccsc1)Nc1ccc(Br)cc1[N+](=O)[O-]', 'O=C([C@H](Nc1ncccn1)c1ccccc1)[C@H]1CCCO1', 'Oc1cccc([C@H]2CNc3ccccc32)c1F', 'Clc1cc(/C=N\\Oc2ccccc2)sc1N1CCCC1', 'Clc1nncc(-n2cccc2)c1C1CC1', 'Clc1nnnn1-c1cc(F)cc(F)c1F', 'Clc1cc(Br)c([N+](=O)[O-])cc1N1CCC[C@H](CO)C1', 'Oc1ccc(F)c([O-])c1-c1ccc([N+](=O)[O-])cc1Cl', 'Clc1ccc(-c2n[nH]cc2-n2ccc([N+](=O)[O-])c2)cc1', 'Clc1cc(/N=C\\CSc2ccc(Cl)cc2Cl)ccn1', 'Clc1ccccc1CCNC1=NC=CCN1', 'Oc1ccc([C@H]([O-])Cc2ccccc2)c(F)c1', 'ClCc1ccnnc1NC[C@H]1COc2ccccc2O1', 'Oc1ccc(C(=O)c2sc(N3CCOCC3)cc2C(=O)[O-])cc1', 'O=C(c1cn2ccsc2n1)N1CCC(Oc2cc(S(C)(=O)=O)nc3ccccc23)CC1', 'Oc1ccccc1OSCC(=O)N1CCOC[C@@H]1Cc1ccccc1', 'Clc1ccsc1-c1nnc(N2CCC(O)CC2)o1', 'COc1nn(CC2CCCC2)c(C)c1S(=O)(=O)N(C)CCC(=O)C(F)(F)F', 'O=C(c1cn2ccsc2n1)N1CC(C2=Cc3ccc(F)cc3C2)C1', 'Clc1ccccc1C1(C[NH2+]Cc2ccccn2)CC1', 'Clc1ccccc1NC1C(=O)NC(=S)N1c1ccccc1', 'Clc1ccccc1OC1(F)CN(Cc2ccncc2)CC1', 'Clc1ccccc1Cc1c(C(=O)[O-])ncnc1NO', 'COCc1n[nH]c(C)c1[C@@H](C)NC(=O)c1ccc(OC)c(F)c1', 'C[C@@H](c1ccccc1)S(=O)(=O)CC(=O)NC(=O)N[C@@H](C)Cc1ccccc1', 'COC(=O)C[C@H]1CCC[C@H]1NC(=O)NCc1cnn(C)c1', 'C[C@@H](OC[C@H]1CCCCO1)C(=O)Nc1ccc([N+](=O)[O-])cc1', 'Clc1ccccc1Cc1c(Cl)ncnc1N[C@@H](C(=O)c1ccccc1)[C@H]1CCCO1', 'O=C(c1ccsc1CO)C(=O)NC[C@@H](c1cc2ccccc2[nH]c1=O)N1CCCC1', 'Clc1cc(NC[C@@H](c2ccccc2)n2cc(S(=O)(=O)N3CCCCC3)cc2)ccc1Cl', 'Clc1cc(Br)c(SCC(F)(F)F)cc1O', 'CCn1nc(C)c([C@@H](NC(=O)Nc2ccc(Cl)c(C#N)c2)c2ccccc2)c1C', 'CC[C@@H](C#N)Oc1ccc(NC(=O)Cn2cc([N+](=O)[O-])cn2)cc1', 'Clc1ccc(Br)c(-n2nc(Br)ccc2=O)c1O', 'Clc1ccccc1-c1csc(/N=C2\\CCCN2CC(F)(F)F)n1', 'O[C@@H](Sc1nnc(C2CC2)n1C1CC1)C(=O)Nc1ccccc1', 'Clc1cccc(F)c1[C@@H]1Oc2c(cccc2Cl)C1', 'Clc1cc(F)ccc1NCC(=O)N1CCC2(CCSCC2)C1', 'O[C@@H](Sc1nnc(-c2ccc(F)cc2)o1)c1cc(Cl)ccc1Cl', 'O=[N+]([O-])c1c(Cl)nsc1Oc1ccc(F)cc1F', 'O=[N+]([O-])c1cc(N2CCN([C@@H]3Cc4ccccc4O3)CC2)ccc1Cl', 'C[C@@H](NCc1ccoc1Br)Cc1cccc(F)c1', 'Clc1ccc(Br)c(NN2C(=O)CSC2=O)c1', 'ClCc1ccnn1Cc1c[nH]c(=O)nc1N1CCCC1', 'Oc1ccc(F)c(S(=O)(=O)[O-])c1', 'Nc1[nH]ncc1CN(Cc1ccccc1[N+](=O)[O-])C[C@H]1CCCO1', 'Clc1ccsc1SCc1ccc(C(=O)[O-])cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(C(F)(F)F)c1C#Cc1ccccc1F', 'Oc1cccc([C@H](NC(=O)Cn2cncn2)c2cccs2)c1', 'Clc1cccc([S@@]c2ncnc(Nc3ccc(Br)cc3Cl)c2[O-])c1', 'Clc1ccc(Br)cc1/N=N\\S(=O)(=O)CCN1CCCC1=O', 'Oc1ccc([N+](=O)C2CC2)cc1', 'Clc1ccccc1Nc1cnc([S-])nc1', 'Cc1sc2nc(NC(=O)C3=CN(c4ccccc4)CC3)sc2c1C', 'ClCc1cc(O[C@@H]2CCc3ccccc32)c(F)cc1F', 'Clc1nnnn1C1CC1', 'O=[N+]([O-])c1c(Cl)n2nnnc2n1-c1ccccc1', 'Clc1cc(COCc2ccc[nH]2)ncn1', 'Oc1cc(OCOc2ccc(Br)cc2)[nH]n1', 'Clc1cc(F)ccc1[C@@H](O)Cc1ccc[nH]1', 'Clc1ccccc1Nc1cnccc1C[NH2+][C@@H]1CCc2ccccc21', 'O=C(c1ccco1)N1CCN(c2cc[nH+]cc2)CC1', 'O=C(c1c[nH]c(=O)[nH]c1=O)N[C@@H]1CCS(=O)(=O)C1', 'Clc1cc(/N=C/CSc2cc(Br)c3ccccc3n2)ccc1O', 'Clc1cc(Br)ccc1NC1(F)CCCCC1', 'Clc1ccccc1Oc1cnccc1C(=O)N1CCCC[C@@H]1C#CCCl', 'O=C(C1CC1)[NH2+]CCCc1ccccc1F', 'O=C(/N=C/N=C/c1cc2ccccc2o1)c1ccc(Br)cc1', 'O=[N+]([O-])c1c(O[C@@H](CO)c2ccncc2)cccc1[N+](=O)[O-]', 'Cc1sn(-c2ccccc2)c(=O)c1NC(=O)c1cc[n+]([O-])cc1', 'Oc1ccc([N+](=O)CCl)cc1N1CCCC1', 'Clc1cc(Br)c2ncccc2c1OC[C@H]1CCCO1', 'Oc1ccc([N+](=O)c2cc(F)cc(F)c2)cc1O', 'O=C(Cc1ncon1)Nc1ncc(Br)cc1Cl', 'NS(=O)(=O)/C=C/CC(=O)Nc1cnn(C)c1', 'O=C(CSCc1n[nH]c2ccccc12)n1cccn1', 'Clc1ccc(-c2nnnn2C[C@]2(O)CCOC2)cc1', 'Nc1[nH]nc(N[C@@H](c2cccs2)c2cccs2)c1Cl', 'Cc1c(Br)cc2c(c1)N(C(=O)[C@@H](C)CCS(C)(=O)=O)CC2', 'O=C(c1ccsc1)N(C[C@H]1COc2ccccc21)c1ccc(C(F)(F)F)cc1', 'Nc1nonc1Cn1ncc2[nH]c3ccccc3c21', 'ClCc1n[nH]c(-c2cncc(F)c2)n1', 'O=C(/C(c1cccs1)Cn1ncc2ccccc2c1=O)c1cccs1', 'Clc1nnnn1c1nc(-c2ccco2)cs1', 'O=[N+]([O-])c1c(Cl)n2nccc2n1Cc1ccccc1Br', 'Clc1ccc(-c2cc(NCc3ccco3)ccc2C[NH+]2CCCC2)cc1', 'O=C(CS(=O)(=O)CC1CCC1)NCC#N', 'Clc1cccc(Br)c1C[C@H](O)c1cnccn1', 'Cc1snnc1-c1cc2cc([N+](=O)[O-])ccc2o1', 'Clc1cc(Nc2nc(C3CC3)no2)c2ncccc2c1F', 'Clc1ccc(-c2nnnn2CC[NH2+]C2CC2)cc1', 'C[C@@H]([C@@H](C)CNC(=O)C1CCC1)NC(=O)c1cccs1', 'Oc1ccc(/C=N/Sc2nnc(N/N=C\\c3ccc(Br)cc3)o2)cc1', 'Cc1sc(C)c([C@H](C)NC(=O)c2ccc(C#N)cc2)c1C', 'Oc1cc(F)ncc1S(=O)(=O)N[C@@H]1CCN(c2ccc(F)cc2)C1', 'Clc1ccccc1C(=O)OC[C@@]1(O)CCSC1', 'Nc1c(F)c(O)c2ccccc2c1OC(=O)CSc1nccs1', 'Oc1ccc([N+](=O)COCc2ccco2)cc1Cl', 'O=[N+]([O-])c1ccn(C(=O)CBr)c1', 'Clc1nnnc(C2=NC(=O)Nc3ccccc32)n1', 'C=CCC[C@@]1(Cc2cccc(OC)c2)CCCN(c2ccc(N(C)C)cc2)C1', 'O=C(Cc1cn(-c2ccccc2)nn1)NCc1cccc(CN2C(=O)[C@H]3CCCC[C@H]3C2=O)c1', 'C=C[C@@H](NC(=O)c1ccc(F)c(C)n1)c1ccccc1Cl', 'Clc1ccc(-c2csc3[nH]c(N4CCCC4)nc23)cc1Br', 'O=C(CNCc1c(F)cccc1F)N1C[C@H](O)[C@H](O)C1', 'O=C(Cc1non2cccc12)N1CCN(CC(=O)/N=C\\c2ccccc2Cl)CC1', 'O=[N+]([O-])c1ccc[nH+]c1-c1nc2ccc(OCC(=O)Nc3c(Cl)cccc3Br)nc2[nH]1', 'Clc1ccc(-c2n[nH]cc2Br)nc1', 'Oc1ccc(F)c(Sc2n[nH]cc2C(=O)NCCc2cccc(O)c2)c1', 'OCc1n[nH]c(NNC(=O)N[C@@H](c2ccccc2)C2CCCCC2)n1', 'Clc1ccc(-c2nonc2)nc1', 'Clc1ccc(-c2nonc2)nc1', 'Clc1ccc([C@@H]([O-])CCc2nc(C3CCCCC3)no2)cc1', 'Clc1ccc(-c2cc(Br)ccc2ONCCCO)c(C(=O)[O-])c1', 'O=[N+]([O-])c1cc(C2SCCS2)c2ccccc2c1[N+](=O)[O-]', 'O=C(/C([O-])(F)F)c1ccc(OC(F)(F)F)cc1OCO[C@@H]1CCc2cc(OCc3ccccc3)ccc21', 'Clc1nnc(Cl)n2ncc(-c3ccc(F)cc3F)c12', 'Clc1ccccc1Oc1cncc(-c2ccccc2)[nH+]1', 'Oc1ccccc1OSC1(CC2CC2)CCOCC1', 'Clc1ccc(-c2cc(/C=N/c3ccccc3)[nH]n2)o1', 'Clc1cccc2nc(/C=C/C3CCCCC3)nn12', 'Clc1ccc(-c2cc(/C=C3\\C(=O)N(Cc4ccco4)C3=O)oc2[O-])cc1', 'O=C(/C=C/O)C(=O)c1ccc(Cl)c(F)c1', 'Clc1ccc(-c2n[nH]c3[nH]cnc23)cc1F', 'Clc1cc(/N=C/c2nc3ccccc3[nH]2)c([N+](=O)[O-])cc1O', 'CCOc1ccc(N2C[C@H](C(=O)NC[C@@H](Br)CCO)CC2)cc1', 'Clc1ccc(-c2nnnn2Cl)cc1O', 'Oc1cc(-n2cnc(NNC(=O)C3CCCC3)c2Cl)cc2c1CCC(=O)N2', 'Clc1ccc(Br)cc1/C=N\\Sc1ccc(C(=O)N2CCCC2)c2nccnc12', 'NS(=O)(=O)/C=C/COc1cccc(F)c1[N+](=O)[O-]', 'Oc1cccc(Br)c1OC[C@H](O)CBr', 'Clc1ccc(Br)cc1-n1cncc1Cn1ccnn1', 'O[C@@H]1Cc2ccccc2CN1C(=O)NCC[C@@H](O)c1ccccc1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1Cl', 'Clc1ccc(-c2csc3[nH]nnc23)cc1Cl', 'Cc1c([N+](=O)[O-])cc(C(C)(C)C)nc1[N+](=O)[O-]', 'Oc1cccc2nc(NC[C@@H](O)Cc3ccccn3)cnc12', 'CC[S@](=O)[C@@H]1CCCC[C@@H]1NC(=O)N1CCC(OCc2ccc(Cl)cc2)CC1', 'CN[C@H](c1cccs1)c1cccc(-n2cncn2)c1', 'Clc1cccc(C(Br)CCl)c1', 'COc1ncc2nc(N/N=C/c3ccc(C#N)cc3)c(CC)nc2c1C', 'Clc1ccc(-c2nnnn2Cl)cn1', 'Clc1ccc(-c2nnnn2Cl)cn1', 'OCc1nn(Sc2ccc(Cl)cc2Cl)c2ccccc12', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1OC[C@@H]1CCCO1', 'Nc1c(Cl)cccc1S(=O)(=O)N[C@@H](c1ccccc1)c1cccc2nc[nH]c12', 'CNS(=O)(=O)c1cc(Cl)c(C(=O)N2CCC[C@@H]2c2cccnc2)cc1Cl', 'O=C(c1cn[nH]c1)N(Cc1cccc2ccccc12)[C@@H]1CCS(=O)(=O)C1', 'Oc1cccc(OC(F)F)c1NC1=NN=C(c2ccccc2)C1', 'Clc1cc(Cl)c(S(=O)(=O)C(F)F)c(Cl)c1Cl', 'O=[N+]([O-])c1cc(C2=NN(Cc3ccccc3Cl)NC2=O)cc(Cl)c1Cl', 'Clc1ccc(-c2cc(OC[C@@H]3CCCCO3)co2)cc1', 'Oc1cc(O)c([N+](=O)[O-])cc1OC[C@H]1CCCO1', 'O[C@@H]1[C@@H](CO)CN(c2ccc(Cl)cc2)C1', 'Oc1cccc(Nc2c(C(F)(F)F)ncnc2/N=C/c2ccccc2Cl)c1', 'O=C([O-])c1[nH]nc(-c2cccc([N+](=O)[O-])c2)c1-c1ccccc1C(F)(F)F', 'COCc1cc(NC(=O)N[C@H]2CC(=O)N(C3CCCCC3)C2)n(C)n1', 'Oc1cccc2cc(OC(F)F)oc12', 'O=C(CS(=O)(=O)[N-]C1CCCCC1)Nc1cccc(C2CCCC2)c1', 'Clc1ccsc1/N=C(\\[O-])COc1ccc(C(F)(F)F)cc1', 'Nc1[nH]ncc1N[C@@H]1CCN(c2ccccc2)C1=O', 'Oc1cc(C=O)ccc1OCCN1CCOC[C@@H]1C1CC1', 'Clc1cccc([S@@]c2n[nH]c(=O)n2C[C@H]2CCCO2)c1', 'COC(=O)[C@@H](C)n1nc(C)c2c(-c3ccccc3)nccc21', 'Clc1ccc(C2=CC[C@@H]2C2CCC2)cc1F', 'Clc1cccc(Sc2nnc(NC[C@@H]3CCCO3)s2)c1', 'ClCc1ncoc1Cn1c(=O)oc2ccccc21', 'Oc1ccc([N+](=O)Cc2coc3ccccc23)cc1OC(F)F', 'O=C([C@H](Sc1ccsc1)c1ccc(F)cc1)N1CCc2ccccc21', 'Nc1ccc(C(=O)N2CCC3(CC2)c2ccccc2NC(=S)N3C)cc1', 'O=C([O-])/C(CCC(=O)Nc1ccc2ccccc2c1)c1ccc(C#N)cc1', 'O=C([C@H](Cl)O)c1ccc(Cl)cc1Cl', 'Oc1ccc(F)c(SCC[NH2+]Cc2ccc(Cl)cc2)c1', 'OC[C@@H]1c2ccccc2C(=O)N1C[C@@H]1CCCO1', 'ClCc1csc(S[C@H]2CCN(Cc3ccccc3)C2)n1', 'Oc1cc(F)c(Cl)cc1N1CC[C@H](N2CCCC2)C1', 'Clc1ccc(C(CO)OC(F)F)cc1', 'Oc1ccccc1O[C@@H](CC(F)(F)F)c1cc(F)cc(F)c1', 'Clc1ccc(Br)c(F)c1COC[C@@H]1Cc2ccccc2S1', 'Oc1ccc([N+](=O)c2onc3c2CCCC3)c2ccccc12', 'Clc1cc(Cl)c(/N=C/c2cc(Cl)ccc2Cl)c(Br)c1O', 'Oc1cccc2nncn12', 'O=[N+]([O-])c1ccn(CC2=CS(=O)(=O)N(c3ccccc3)c3ccccc32)c1', 'O=C([O-])c1[nH]nc(-c2ccc([N+](=O)[O-])cc2)c1Br', 'COc1nn(C)cc1CN(C)C(=O)N[C@@H]1CCN(c2cnn(C)c2)C1', 'Oc1ccccc1C1CCN(C(=O)CN2c3ccccc3CC2)[C@H]2CS(=O)(=O)C[C@@H]21', 'O=[N+]([O-])c1cc(SC[C@H]2CC=CCC2)ccc1O', 'Clc1cc(N2CCC[C@@H]2c2nccs2)ccc1F', 'O=C(c1cncc(Br)c1)N1CCN(c2[nH+]ccn2Cc2ccccc2)CC1', 'Clc1cccc(/N=C/c2cc([N+](=O)[O-])ccc2OC[C@@H]2CCCO2)c1', 'Cc1nonc1N(C)C(=O)[C@@H](C)Oc1ccccc1Cl', 'Clc1ccc(Br)cc1[C@@H]1C[C@H]1c1ccccc1F', 'Clc1ccc(-c2cccnc2N2[C@@H](O)CCC[C@H]2OCc2ccccc2)cc1', 'Clc1ccccc1N1Nc2c(ccc(Br)c2F)C1', 'O=C(CS(=O)(=O)[C@@H](c1ccccc1)N1CCOCC1)Nc1ccc2c(c1)OCCO2', 'Nc1nonc1C(=O)Nc1sccc1C(F)(F)F', 'CNc1n[nH]c(=S)n1CC(=O)N1CCO[C@@H](c2ccccc2)C1', 'COC[C@@H](NC(=O)OC(C)(C)C)C(=O)NN', 'O=C([C@H](C(=O)NC1CC1)N[C@@H]1CC(C)(C)c2ccccc21)N1CCN(c2ccc([N+](=O)[O-])cc2F)CC1', 'Oc1ccc(O[C@@H](C(=O)[O-])c2cccc(Cl)c2)cc1', 'O=C(c1c[nH]c(C(=O)N2CCC[C@H](c3nc4ccccc4[nH]3)C2)n1)N1CCCC1', 'COc1nn(-c2cccc(C(N)=O)c2)c(C(=O)[O-])c1C', 'O=C(c1cncc(F)c1)N1CCc2sccc2[C@@H]1c1cccc(Br)c1', 'Clc1cc(NSc2ccc(Cl)nc2Cl)ccc1[N+](=O)[O-]', 'O=C(/N=C\\NC1=NC=NS1)c1csc(-c2ccc(F)cc2)n1', 'Clc1nnn(-c2ncnc(-c3ccncc3)n2)c1Cl', 'ClC(/C=C(/C#N)c1ccccc1)Nc1ccccc1', 'O=C(Cc1c[nH]nc1-c1ccccn1)N[C@H](CO)c1ccccc1', 'O=C(/C(c1cccs1)C(=O)Nc1ccc(F)c(F)c1)c1cccs1', 'N[C@@H](Cc1ccc2c(c1)C(=O)CCC2)c1cc(OC(F)(F)F)ccc1Cl', 'Nc1nccnc1SCC(=O)N[C@@H]1CCS(=O)(=O)C1', 'Clc1nncc(Oc2nc(-c3ccccc3)no2)c1I', 'O=[N+]([O-])c1c(Br)c(O)nc2ncccc12', 'Nc1[nH]nc(CCC(=O)N[C@@H]2CCCC[C@@H]2Cc2ccccc2)n1', 'CN(CCc1cccc(Cl)c1)C(=O)c1cnc([C@@H]2CCCO2)nc1C', 'Clc1ccccc1[C@H]1Nc2c(cccc2Cl)C(=O)N1C1CCCCC1', 'ClCc1ccccc1O/N=C1/N=C(c2ccccc2)SC1=O', 'O=C(c1cn(C2CC[NH2+]CC2)nn1)N1CCN(c2ccc(F)cc2)CC1', 'Nc1[nH]nc(-c2ccco2)c1N', 'O=C(CNS(=S)(=O)c1ccc(Br)cc1)Nc1cccc([N+](=O)[O-])c1', 'Nc1c(Cl)n2ncnc2n1CCS(=O)(=O)CC(=O)N1CCCC1', 'ClC[C@H](CS(=O)(=O)c1ccc(Br)c(F)c1)NC(=O)c1ccc2c(c1)OCCO2', 'Oc1ccccc1-n1nnnc1SCC(=O)N1CCC(C2c3ccccc3C2)CC1', 'Clc1ccccc1Oc1cnc(C[NH+]2CCC(c3noc4ccccc34)CC2)cn1', 'O=C(/C=C\\N=C1\\Nc2ccccc2C1=O)c1ccccc1Cl', 'Clc1cccc(CC/C([O-])=C2/NC=C/c3ccccc32)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])c(F)cc(Oc2cncc([N+](=O)[O-])c2)c1C1CC1', 'Clc1ccc(C[NH2+]Cc2cc(Br)ccc2O)cc1', 'ClC(/C=N/c1cccc(Br)c1)c1ccccc1', 'Clc1cc(Br)c2nc(-c3cc[nH]c3)[nH]c2c1', 'Clc1ccc(Br)c(Cl)c1C1CC[NH+](Cc2ccccc2)CC1', 'O=C(c1c[nH]nc1-c1ccc2c(c1)OCO2)N(C1CCCCC1)S(=O)(=O)c1ccccc1', 'Oc1ccccc1CC[NH+]1CCN(Cc2ccccc2)CC1', 'ClCc1csc2nc(SCc3cccnc3)nnc12', 'Clc1nnnn1/N=C/C(=O)c1ccc(F)cc1', 'Clc1ccc([C@@H](CCc2nc([C@H]3CCCN3c3ncc[nH]3)no2)c2ccccc2)cc1', 'CN=C(C#N)CC1(c2ccc(OC)cc2)CC1', 'Clc1cc(N2CC[C@H](Nc3ncccn3)C2)ncc1Cl', 'ClC[C@H](c1ccc(Br)s1)N1CCC2(CC1)COc1ccccc12', 'Oc1cc(-n2nnc(C3CC3)n2)c(Cl)cc1Cl', 'N#Cc1c(NC(=O)C=CCCS)c2ccccc2n1C', 'CNC(=O)c1cc(S(=O)(=O)N2CCC[C@@H]2C2CCCCC2)cc2c1OCCO2', 'Clc1nnn(-c2ncc(C(F)(F)F)cc2Cl)c1Br', 'N=C1C=C(C(=O)N[C@H](c2ccc(OCN3CCOCC3)cc2)c2cccs2)c1', 'ClCc1ncnn1-c1ncnc2c1CCCC2', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(C(F)(F)F)c1C[NH2+]Cc1ccccc1', 'CN=C(/C#N)c1cccc(C#N)c1', 'Cc1cc(F)ccc1CCN(C(=O)[C@H](C)[N+](C)(C)C)c1ccccc1', 'Cc1ccsc1/C=C/C(=O)N1CCC(C(=O)[O-])CC1', 'O=C([O-])/C(CO)CCn1nnc2ccccc21', 'Clc1ccccc1[C@H]1C[C@H](CCCO)CN1C(=O)c1cc2c([nH]c1=O)CCC2', 'O=C(c1cc2ccccc2oc1=O)N1CC[C@@H](n2cncn2)C1', 'Clc1ccc(CN2C[C@@H](c3n[nH]c(=O)[nH]3)CC2)c2ccccc12', 'O=C(c1cc(Cl)c2nccnc2n1)N1CCC[C@H]1c1ccc(F)cc1', 'O[C@@H](Cc1cccc(Br)c1)[C@@H]1CCCO1', 'Clc1ccc(C[NH2+]Cc2cc3ccccc3o2)cc1', 'O=C(/N=c1\\o/c(=C\\c2ccc(F)cc2)[nH]c1=O)c1c(Cl)cccc1Cl', 'N#C/C(=C\\c1cc(Cl)ccc1OCCO)C(F)(F)F', 'Clc1cccc2ncn(C(Cc3ccco3)c3ccccc3)c12', 'Oc1cc(Br)c(Cn2ccnn2)c(Cl)c1', 'Clc1cccc2cc(Oc3nc(-c4ncn[nH]4)cs3)ncc12', 'N[C@@H]1Oc2ccc(C(=O)Nc3ccccn3)cc2C(=O)N1c1ccccc1Cl', 'NS(=O)(=O)Cc1noc(-c2sncc2-c2ccccc2)n1', 'O=[N+]([O-])c1c(Br)cnc(Oc2ccccc2Cl)c1C(F)c1ccccc1', 'Cc1ccsc1[C@@H](N)Cc1ccc(Cl)c(F)c1', 'ClC(=S)c1ccnc(C2CC2)n1', 'OC[C@@]1(Cc2cccnc2)Oc2ccccc2C1', 'OC[C@@H](C[C@@H]1CCCO1)N(CC(=O)Nc1ccccc1)c1ccccn1', 'Oc1ccc([N+](=O)[O-])c(Br)n1', 'O=[N+]([O-])c1c(Cl)nsc1Nc1ccc([N+](=O)[O-])c([N+](=O)[O-])c1', 'O=C([O-])/C=C/c1cnn(-c2ccc(Br)cc2)c1N', 'O=C(c1c[nH]nc1-c1cccs1)N1CC[C@@H](Oc2ccccc2)C1', 'C=C[C@@H](O)CN1CCc2ccc(C(=O)NC)cc2C1', 'Oc1ccc(/C=N\\c2ccccc2)c[nH+]1', 'Clc1ccccc1N1N=CN(S(=O)(=O)c2ccc(Cl)c(Cl)c2)CC1', 'Cc1nonc1CCCC(=O)NNC(=O)C[C@@H](O)c1ccccc1C', 'Clc1cccc(/N=N/[C@H]2c3cc(OCc4ccccc4)ccc3C2=O)c1', 'Clc1ccccc1Oc1cc(SCC(=O)[O-])cc(Cl)n1', 'OCc1nnsc1N1CCOCC1', 'ClCc1nonc1-c1cc(Br)cc(Br)c1', 'OCSCCOCCc1n[nH]c2cc(OC(F)F)ccc12', 'N=Cc1nonc1CSCc1ccccc1', 'Oc1ccc(F)c(-c2nnc(NC[C@@H]3CCCO3)s2)c1', 'Clc1nncc(-n2ccnc2)c1NC1CC1', 'Clc1cccc2[nH]nc(NC[C@@H](c3ccsc3)c3ccccc3)c12', 'Clc1nnc(-c2ccccc2)nc1CS[C@@H]1CCCc2ccccc21', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n\n', 'Clc1ccsc1F\n\n\n\n', 'Clc1ccsc1F\n\n\n', 'Clc1ccsc1F\n\n', 'Clc1ccsc1F\n', 'Clc1ccsc1F', 'Clc1ccsc1F', 'O=C([O-])/C(=N/c1ccccc1)c1ccc(Br)o1', 'O=C(c1ccsc1)N(C1CC1)[C@@H](CCCn1cncn1)c1ccc(F)cc1', 'O=C(c1cncc([N+](=O)[O-])c1)N1[C@@H](CO)CC1(c1ccccc1)c1ccccc1', 'Oc1ccccc1[C@H]([C@@H](O)c1ccncc1)NC1CC1', 'O=C([C@H](Cc1ccccc1)N1CCOCC1)[C@H]1CN2CCCC[C@@H]2CO1', 'Clc1nncc(O[C@H]2CCN(c3ccccc3Cl)C2)n1', 'O=C([C@H]1CC[C@@H](C[NH3+])C1)[C@H](Cc1ccccc1)N1C(=O)c2ccccc2C1=O', 'O=C(Cc1noc(-c2ccncc2)n1)N[C@@H]1CCS(=O)(=O)C1', 'ClC(=S)[C@@H](NC(=S)Nc1cc(F)ccc1F)c1ccc(Cl)c(Cl)c1', 'Clc1cccc(N2CC[C@@H](C(=O)[O-])C2)c1', 'O=[N+]([O-])c1ccc(N[C@H]2CCC[C@H](S(=O)(=O)CCc3ccccc3)C2)cc1', 'Clc1cccc(S(=O)(=S)Nc2ccccc2C(F)(F)F)c1', 'Clc1cccc(F)c1C1CC[NH+]([C@@H]2CCN(c3ccccc3)C2)CC1', 'Clc1nnn(-c2ncccn2)c1Br', 'Oc1ccccc1[C@@H](c1cccnc1)N1CCN(S(=O)(=O)c2ccc(Cl)cc2F)CC1', 'Clc1nnc([S-])n1CCOC(=O)c1ccc(Cl)c(F)c1', 'O=[N+]([O-])c1c(F)cccc1[C@@H](O)c1cc(F)ccc1F', 'Clc1cc(NSc2nc3ccccc3[nH]2)ncn1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnnc1CSC1CCC1', 'NS(=O)(=O)N1CCC[C@H]1c1cc(F)cc(F)c1', 'CN=C/[NH2+]Cc1ccc(-c2ccccc2)[nH]c1=O', 'COc1nc(N[C@@H](COC(C)C)Cc2ccccn2)ccc1C', 'O[C@@H]1C(c2ccc([N+](=O)[O-])cc2)=C(O)c2ccccc21', 'O=C([C@H](Cl)CCSc1ccccc1)NC[C@@H]1CCCO1', 'Clc1ccccc1OCC1CC[NH+]([C@@H]2Oc3ccccc3O2)CC1', 'Clc1ccc(-c2nonc2[S-])cc1', 'Clc1ccc(-c2nonc2[S-])cc1', 'Clc1ccc(-c2nonc2[S-])cc1', 'Clc1ccc(-c2nonc2[S-])cc1', 'Oc1cccc(SCS(=O)(=O)c2ccsc2)c1', 'Clc1ccccc1O/N=C(/OCc1cccc([N+](=O)[O-])c1)n1cncn1', 'Clc1ccc(Br)cc1O[C@H](c1ccccc1)[C@@H]1CCS(=O)(=O)C1', 'Clc1nnn(-c2nccc(Cl)c2Cl)c1NS(=O)(=O)c1cccc(F)c1', 'O=C(CS(=O)(=O)[N-]c1ccccc1C(=O)c1ccc(F)cc1)NNC(=O)c1cccs1', 'O=C(c1cc(Br)cc(Cl)c1)[C@@H]1CCOC1', 'Oc1cc(O)c2ccc(Cl)cc2c1OC(F)c1ccccc1', 'Cc1sc(-c2cccs2)cc1[C@H](C)NC(=O)Nc1ccc(C#N)cc1', 'O=[N+]([O-])c1c(Cl)n2nnc(-c3ccncc3)c2cc1Cl', 'ClC(c1cc2c(cc1Cl)OCCO2)c1ccc(F)cc1F', 'O=C(c1cc([N+](=O)[O-])on1)N1CCC[C@@H]1c1ccccc1', 'Clc1ccc(Br)cc1O[C@H](N)c1ccc(F)cc1F', 'Clc1ccsc1/N=C\\C(=O)NCCN1C(=O)/C(=C/c2ccc(Cl)cc2)SC1=S', 'Clc1cccc(-c2n[n-]c(-c3ccccc3)n2)c1', 'O=C(CNc1cc2[nH]c(=O)[nH]c2cc1Br)N1CCC[C@@H]2CCC[C@H]21', 'O=C([C@H]1c2cc(O)ccc2Oc2ccc(F)cc21)NC1CCCC1', 'O=C(/C=C/c1c(N[C@@H]2CCSc3ccccc32)[nH]c2ccccc12)NCc1ccc(C(=O)OCCO)o1', 'O=C(C1CC[NH+]([C@H]2CCO2)CC1)N1CCN(C(=O)Nc2ccccc2)CC1', 'N#CCOc1ccc(/C=N/c2c[nH]cn2)cc1', 'O=C(c1cn2ccsc2n1)N1CCC[C@@H]1c1cccc(Cl)c1', 'ClC[C@H](CC1CCN(C(=O)c2cccnc2)CC1)n1cccn1', 'ClCc1ccnnc1-n1cncn1', 'C=CCn1cc(NC(=O)c2[nH]nc(C3CC3)c2C)c(C)n1', 'O=[N+]([O-])c1c(OC[C@H](O)COc2cccc(F)c2)nccn1', 'Clc1cccc2c1Nc1nnc(SCc3ccc(Br)s3)nc12', 'ClC[C@H](OCCO)C1CCN(C(=O)c2ccco2)CC1', 'ClC[C@@H]1C[C@H]1c1cccc(Cl)c1', 'Nc1cn[nH]c1[C@@H]1CCCN1C(=O)C1CCCC1', 'Clc1cccc2[nH]nc(S/C=C/C3CCCCC3)c12', 'ClCc1ccn(C[C@@H](O)c2cccnc2)c1', 'Clc1ccsc1[C@@H](Cl)CCOc1ccccc1F', 'O=C(c1cccn1Cc1ccccc1)N1CCc2[nH]c[nH+]c2C1', 'CN[C@H]1[C@@H](OCc2ccccc2)N(Cc2ccccc2F)C1=O', 'N[C@@H](C#N)c1ccccc1Cl', 'Oc1cccc(Sc2nnnn2-c2c(C(=O)[O-])cnn2-c2ccc(Cl)cc2)c1', 'Oc1cccc2cc(CN3CCC[C@@H]3C(=O)c3cccc(C(F)(F)F)c3)oc12', 'Clc1nnnc(CN2CCC[C@@H]2c2ccccc2)n1', 'Clc1ccc(Br)cc1/C=N\\C(=O)C1=NN=C1Cc1ccccc1', 'Clc1cccc2ncnc(N3CCO[C@H]4CCCC[C@@H]43)c12', 'O=C(c1cccc(S(=O)(=O)[N-]c2ccc(Cl)c(Cl)c2)c1)N1CCCCC1', 'OCc1ncco1', 'CNC(=O)N[C@H]1CC(=O)N2c3ccccc3C(=O)N2[C@H](c2ccccc2)[C@@H]1C(=O)c1cccc(Br)c1', 'OC[C@H]1CN(Cc2ccccc2CO)C[C@@H]1c1ccco1', 'Clc1ccsc1N[C@@H]1CCCC[C@H]1COc1ccccc1F', 'Cc1sncc1CNc1ccnc(OCC2CC2)c1', 'Oc1cccc(-n2c(S[C@H]3CCOC3)nc3ccccc32)c1', 'Clc1nnc(Cl)nc1C1CC1', 'Clc1ccsc1-c1nc(Cl)ncn1', 'O=C(c1ccsc1)[C@@H]1CC(c2ccc(Cl)cc2)=NO1', 'Oc1cc(O)c(Oc2ccc(N3CCCC3=O)cc2)nc1NC[C@@H]1CCCO1', 'O=[N+]([O-])c1c(OCc2nsc(-c3cccs3)n2)oc2ccccc12', 'O=[N+]([O-])c1cc(F)cc(CN2CCOC[C@H]2Cn2cnnn2)c1', 'NS(=O)(=O)/C=C/c1cccnc1Cl', 'O=C([C@H](NS(=O)(=O)c1ccc([N+](=O)[O-])cc1)c1cccs1)Nc1ccccc1C(F)(F)F', 'O=C(c1c[nH]c2cc([N+](=O)[O-])ccc12)[C@@H](Cc1ccccc1)n1cccn1', 'Nc1c(Cl)cc(/C=C/Cl)c(Cl)c1', 'COC[C@@H]([NH2+]CC1CCN(Cc2ccccc2F)CC1)C(=O)OC', 'Clc1ccc(Br)cc1CC/N=c1\\sc(-c2cccc(Cl)c2)n[nH]1', 'O=C(c1cccnc1N1CC[NH+](Cc2ccc(Cl)cc2)CC1)N1CCCCC1', 'NS(=O)(=O)/C(=N/O)c1cccc([N+](=O)[O-])c1', 'Oc1ccc([N+](=O)COc2ccccc2C(=O)[O-])cc1', 'Clc1cc(F)c(/C=N/NC(=O)c2ccco2)s1', 'Clc1ccc(Br)cc1O[C@H]1CCCN(C(c2ccccc2)C(F)(F)F)C1', 'Clc1cc(Br)c2nc(-c3cccs3)[nH]c2n1', 'O=C(CCc1cc[nH+]cc1)Nc1cccc(CNc2ncccn2)c1', 'Clc1ccc(Br)cc1[C@@H]1Oc2ccccc2C(=O)N1CCCC(F)(F)F', 'Clc1ccccc1O/N=C(/Cl)c1nccs1', 'Clc1ccccc1-c1cscc1C[NH3+]', 'Nc1nnc(SCC(F)(F)F)n1N', 'Oc1cc(Br)cc2c(Cl)cc(SCc3ccoc3)nc12', 'COc1nn[nH]c1C(=O)NC[C@@H](CCO)c1ccccc1', 'Cc1nonc1Cc1nc(C)c(C)s1', 'O[C@@H]1[C@@H](CC(=O)NNc2cnc3ccccc3n2)CCN1c1ccccc1', 'Oc1ccc([N+](=O)c2ccc(Br)o2)cc1', 'Cc1cc([C@@H]2CCCCN2c2ncnc3[nH]ccc23)nc2ccccc12', 'Oc1ccc(O[C@H]2CCOC2)cc1-n1cccn1', 'Clc1cc(Cl)nn1/N=C\\c1ccccc1Cl', 'Clc1nncn1c1cc(Cl)ccc1N1CC[C@H](CN2CCOCC2)C1', 'Clc1ccccc1Nc1nncc([O-])c1[N+](=O)[O-]', 'O=[N+]([O-])c1c(Cl)nnc2[nH]nc(-c3ccc(F)c(F)c3)c12', 'Clc1ccsc1/N=c1\\sccn1Cc1cccc([N+](=O)[O-])c1', 'O[C@@H]([C@@H](n1cc(C(F)(F)F)c2ccccc21)C1CCCC1)N1CCOCC1', 'Oc1cc(O)c2[nH]cnc2c1/N=N/c1ccccc1', 'Clc1cc(NSc2nnnn2C2CCCC2)ccn1', 'ClCc1ccnnc1Cl', 'Oc1cccc(SC[C@H]2C[C@H]2c2ccc(Cl)cc2)c1', 'OCc1nn(Cc2ccco2)c2[nH]ccc12', 'Clc1nnnn1c1[nH]c(C2CC2)nc1N1CCc2ccccc21', 'CCS(=O)(=O)c1cc(C(=O)[C@@H](C#N)c2ccncc2)cc(OC)c1OC', 'N=Cc1c([N+](=O)[O-])cccc1C(=O)NC[C@@H](O)C(=O)c1ccccc1', 'Clc1ccccc1CNC[C@H]1CCC[C@H](CNS(=O)(=O)C2CC2)C1', 'Clc1cc(/C=C(\\CCl)C2CC[NH+](Cc3ccccc3F)CC2)cc2[nH]ccc12', 'Clc1ccc([C@@H](NC2COC2)C(F)(F)F)cc1', 'OC[C@H]1[C@H](Cc2ccc(O)cc2)NC(=O)N1Cc1ccc(F)cc1', 'Clc1cccc([C@@H](c2cccs2)COc2ccc([N+](=O)[O-])cc2)c1O', 'Clc1cc(Nc2nc[nH]n2)ccc1O', 'Clc1ccc(-c2csc3s/cnc23)cc1O', 'Clc1cc(Cl)nnc1N/C=C/c1ccc(CCCO)o1', 'COC[C@H](NCc1ccccc1C)[C@H]1CCOC1', 'O=[N+]([O-])c1ccncc1O/N=C1\\C(=O)Nc2ccccc21', 'Clc1ccc(-c2nonc2)cn1', 'Clc1ccc(-c2nonc2)cn1', 'O=C(c1cnn2cccnc12)N1CCC[C@H]1c1cccs1', 'Clc1ccc(-c2csc3[nH]ncc3c2=O)cn1', 'O=C(Cc1ccccc1)NC[C@@H](c1cccs1)[C@H]1CCCO1', 'Clc1cc(Br)c2ncnn2c1Cl', 'NS(=O)(=O)Nc1ccc(C(=O)OC(C(C)C)n2ccnc2)cc1', 'Clc1cc(Br)c(Nc2nc([O-])c3ccccc3n2)c(O)c1', 'ClC[C@@H]1OCC[C@H](Sc2cccc(Cl)c2)C1', 'Clc1cccc2cc(OC[C@@H]3CCCO3)oc12', 'Clc1ccccc1N1N(Cc2ccccc2)C[C@@H]2CCCC[C@@H]21', 'COc1ncc(C)cc1OC(=O)[C@@H]1Cc2ccccc2CN1C(=O)C(C)C', 'O=C(c1cn[nH]n1)N(Cc1ccsc1)c1nc(C(F)(F)F)cs1', 'Clc1cccc2cc(Br)c(/C=C/C(=O)[O-])cc12', 'O[C@@H](Sc1ccncc1)c1cccc(Br)c1', 'Clc1nncnc1N[C@H](CO)c1ccccc1F', 'Oc1ccccc1/N=N/n1c2ccccc2c2ncnn21', 'Oc1ccc(/N=c2/[nH]c(C(F)(F)F)ccc2-c2cccc(Br)c2)cc1[N+](=O)[O-]', 'O=C(CSCC(F)(F)F)N1CCC[C@](N2C(=O)c3ccccc3C2)C1', 'Clc1ccc(Br)cc1O[C@@H](Nc1ccc(F)cc1)n1cccn1', 'O=[N+]([O-])c1ccncc1[C@@H]1c2ccccc2C(=O)N1CN1C(=O)c2ccccc2C1=O', 'O=C(c1cnc(Cc2ccccc2)s1)N1CCC2(CC1)Nc1ccccc1C2=C1CCOCC1', 'Clc1ccc([C@@H]([NH2+]CC(=O)c2ccc(F)cc2)C2CCCC2)cc1', 'OC[C@H](CCS(=O)(=O)c1ccccc1)N[C@H]1CC(=O)N(c2ccccc2)C1', 'O=[N+]([O-])c1ccc2onc(Nc3ncccc3Cl)c2n1', 'Oc1cc(F)nc(Sc2cc(Br)ccc2O)n1', 'N[C@@H](CC1CCOCC1)C(=O)Nc1ccsc1', 'Clc1cc(/N=C\\c2ccc(F)cc2F)c2[nH]cnc2n1', 'Oc1ccc(O[C@H](F)c2cccc([N+](=O)[O-])c2)cc1F', 'Oc1cc(-n2nnc(-n3cccn3)n2)ccc1F', 'NS(=O)(=O)NCc1cc(C2CC2)ccc1[O-]', 'Clc1ccc([C@@H](Br)C[C@H](c2ccc(Cl)cc2)[NH+](C)C)cc1', 'O=C(c1cn[nH]c1-c1ccc(Cl)cc1)N1CCN(C(=O)N[C@@H]2CCSc3ccccc32)CC1', 'Oc1ccc([C@H]2NC(=O)N/C(=C\\c3cccc(O)c3)N2Cc2ccccc2)cc1', 'Clc1ccc(Br)c([C@@H]2OCCc3ccc(F)cc32)c1', 'O=C([C@H](c1ccccc1Br)N1CCOCC1)N1CCC(c2ccncc2)(C(=O)[O-])CC1', 'N=Cc1nonc1Cc1ccccc1F', 'Oc1cccc2cc(/C=N/N=C\\c3ccco3)oc12', 'COc1nnnn1CCC(=O)N(Cc1ccc[nH]1)[C@@H](C)c1ccccc1', 'O=[N+]([O-])c1c(Br)scc1C(=O)Nc1cccs1', 'Oc1cc(-n2nnnc2Sc2ccccc2Cl)cs1', 'ClCc1cccc2c1O[C@@H](Cc1cc(Br)ccc1OC[C@@H]1CCCO1)C2', 'O=C(CN(c1cnccc1)S(=O)(=O)[O-])C1CCCC1', 'Clc1ccc(-c2csc3s/cnc23)c(Br)c1', 'Clc1ccc(COc2nc([O-])c(C(=O)[O-])cc2Cl)cc1', 'Oc1cc(-n2cnnn2)c2cnn(CC3CCCCC3)c2n1', 'O[C@@H]1C(C(=O)OCc2ccccc2)c2ccccc2S1', 'Clc1nnnn1-c1c(-c2ccccc2)c[nH]c1CSc1nc2ccccc2n1CC(F)F', 'C[C@@H](CC(=O)Nc1ccc2c(c1)OCO2)CCC(C)(C)C#N', 'Clc1cc(Cl)nn1/N=C\\c1ccc2c(c1)OCO2', 'Clc1nnc(CSc2c[nH]c3ccccc23)[nH]1', 'COc1nnn(-c2cc(N)ccc2Sc2ncccn2)c1C', 'Clc1cc(/C=C2\\S/C(=C/c3ccc(O)c(OCc4ccccc4)c3)C(C#N)=C2[O-])ccc1O', 'O=C(COc1cc(C(F)(F)F)sc1-n1ccnc1)c1ccc(C#N)cc1', 'O=[N+]([O-])c1c(Cl)nc(Cl)cc1Br', 'Clc1ccccc1Cc1cccc(OC2CC[NH2+]CC2)c1', 'ClCc1n[nH]c(C2CC2)n1', 'Clc1ccc(-c2cc([C@H]3CCCO3)n[nH]2)cc1', 'O=[N+]([O-])c1ccnc2c1NCC2', 'O=[N+]([O-])c1c(O)sc2c1nnn2-c1ccccc1', 'Clc1nnnn1CC(=O)N(Cc1ccccc1)[C@H]1CCS(=O)(=O)C1', 'Clc1ccsc1-c1nnco1', 'Clc1cccc(-c2ncc(CSCc3n[nH]c(-c4ccccc4)n3)n2Cn2cncn2)c1', 'O=C(c1c[nH]c2cc(F)ccc12)N1CCC(O[C@@H]2CCS(=O)(=O)C2)CC1', 'Clc1ccccc1OCc1ccncc1COCC1CC[NH+](C)CC1', 'O=[N+]([O-])c1ccn(Cc2cc(F)cc3c2OCOC3)c1', 'Cc1cnc2oc(C)c(C(=O)N[C@H]3CC(=O)N(c4ccccc4F)C3)c2c1', 'O=[N+]([O-])c1ccc([C@H]2C[C@H]2CN2CCOCC2)cc1', 'O=C(/N=C/c1cc2ccccc2[nH]1)C1(c2cccnc2)CC1', 'Clc1cccc([S@@]c2ccc([O-])nn2)c1', 'ClC[C@@H]1COCCN1c1cccc(O)c1', 'O=C([C@H]1C[C@H]1c1cccs1)N1CCN(c2ccc(F)c(F)c2)CC1', 'O=C(c1ccco1)c1cc(S(=O)(=O)NC[C@H]2CCCO2)c(F)cc1F', 'Clc1ccc(Br)c(N2CCN([C@@H]3CCCC[NH+]3Cc3ccccc3F)CC2)c1', 'O=C(c1cnn2nc(C3CC3)oc12)N1CCC([C@H]2CCOC2)CC1', 'Oc1cc(F)ncc1[C@H](O)c1cccc(F)c1', 'Cc1nonc1C[C@H](CS)Cc1ccccc1', 'O=C([C@H]1c2cc(F)ccc2OC1)N1CCN(c2ccc(C(F)(F)F)cn2)CC1', 'OC[C@H](Cn1ccc(=O)[nH]c1=O)Nc1ccc2c(c1)OCO2', 'Oc1cc(F)ncc1C(=O)NNc1ncc(Cl)c(Cl)c1Cl', 'Clc1ccc(Br)c(Sc2nc(=O)n3ccsc3n2)c1', 'O=[N+]([O-])c1c(-c2nsc(-c3ccsc3)n2)nc2ccccc2c1[O-]', 'OC[C@@H](Sc1nncs1)C(=O)Nc1cccc2ccccc12', 'NS(=O)(=O)/N=c1\\sc(-c2ccc(F)cc2)cc(-c2ccco2)n1', 'O=C([C@H](Sc1ncccn1)C(=O)NC1CC1)NCc1cc(F)ccc1F', 'Clc1cc(Cl)nn1/N=C\\c1cccc(Br)c1', 'O=[N+]([O-])c1ccc2c(c1)[C@H]1C=CC[C@@H]1[C@@H](c1cccc(OCc3ccccn3)c1)N2', 'NS(=O)(=O)C[C@@H]([N+](=O)[O-])c1cccc2ccccc12', 'O=C([C@H](Nc1nc2c(Cl)cccc2s1)c1cccs1)Nc1c(F)cccc1F', 'ClC(c1ccccc1)[C@@H]1CCS(=O)(=O)C1', 'COc1nc(CN(C)c2nc(C)nc3c2c(C)nn3-c2cccc(C)c2C)cs1', 'Clc1cc(NC2(c3ccc(F)cc3)CCOCC2)cs1', 'Clc1ccc(Br)cc1C[C@@H]1CCCN1c1nc(F)ccc1F', 'Nc1[nH]nc2nc(SCc3ccccc3C(F)(F)F)cnc12', 'O=C(c1c[nH]nc1-c1ccccc1)N1CC[C@@H](c2ccccc2)[C@H]1Cc1cccs1', 'Clc1cc(CNC[C@H](N)c2ccccc2)[nH]n1', 'NS(=O)(=O)/C(=N/Nc1ncccc1Cl)c1cc(F)ccc1O', 'Oc1ccc([N+](=O)[O-])c(C[NH2+]Cc2ccccc2Br)c1', 'Clc1ccc(Br)cc1-c1cnnn1C(F)(F)F', 'Clc1cccc2ncn([C@@H](CO)C(=O)N3CCCC3)c12', 'Oc1cccc(F)c1SCC(=O)N1CSC[C@@H]1c1ccccc1', 'Clc1nnnn1c1nnnn1C1CCCC1', 'Cc1sn(-c2ccccc2)c(=O)c1S(=O)(=O)N1CCC[C@H]1Cc1ccccc1', 'COC(CC)(NC(=O)c1ccc(C)c(C)c1)C(=O)OC', 'O=C([O-])/C(=N\\O)c1ccccc1Br', 'O=C([O-])c1nn(C(=O)c2ccc(Br)cc2)c2ncc(C(F)(F)F)cc12', 'Clc1ccc([C@@H]([O-])CCCNc2ccccc2F)o1', 'N=Cc1ncnc2cc(-c3cc(C)cc(OC)c3)nn12', 'ClC(c1cc(Cl)ccc1Cl)N1CCc2ccc(F)cc2C1', 'Clc1ccc(-c2csc3[nH]c(N[C@@H]4CCSCC4)cc23)cc1', 'Clc1cc(F)c(C[C@@H]2CCC[C@@H]2O)cc1', 'O=C(/C(c1cccs1)=C1/N=C(c2ccc(Br)cc2)SC1=S)c1ccccc1', 'Clc1ccccc1C1=CN(c2ccc3nnnn3n2)CC1', 'O=C(/N=c1/[nH]cccc1Nc1ncccn1)c1cccs1', 'O=[N+]([O-])c1cc(Cl)nc([N-]SCc2ccc(OCc3ccccc3)cc2)c1', 'Clc1ccc(-c2cccs2)nc1C[C@H]1CCCCN1', 'Clc1cc(NSc2nnc(-c3ccccc3)n2Cc2ccco2)n[nH]1', 'CC[C@@H]1CN(C(=O)NCCNC(C)C)CCO1', 'Oc1cc(-n2nc3sc4ccccc4nc32)cs1', 'O=C([O-])CS(=O)(=O)Cc1nccn1Cc1c(F)cccc1Cl', 'Clc1cc(C2OCCC2)ccc1-n1cnnn1', 'Clc1ccccc1NS[C@H](O)c1ccc(Br)cc1', 'O=[N+]([O-])c1cc(Cl)ncc1NC[C@H](SCc1ccccc1)c1ccccc1', 'Clc1cccc(CNC[C@H]2CCC[C@@H]2n2cccn2)c1', 'N#C[C@@H](C(=O)c1ccc(CN2CCSCC2)cc1)c1ccoc1', 'NS(=O)(=O)CC/C=C(\\C=C\\C=C/c1ccc(Br)cc1)NC(=O)c1ccccc1Cl', 'Nc1[nH]nc2c1CCCC2', 'ClCc1cc(OC[C@H]2CCCO2)nc(OC2CCCCC2)n1', 'ClCc1ccnnc1SCC(=O)NC[C@@H]1CCCO1', 'Clc1cccc2ncn(C[C@H](O)c3ccco3)c12', 'O=C(c1cn(Cc2ccco2)c2c1CCCO2)N1CCSCC1', 'O=C(CSCNC(=O)c1ccc([N+](=O)[O-])cn1)Nc1cnc(-c2cccnc2)o1', 'O=C(C1=Cc2cc(F)ccc2OC1)N1CCCc2ccsc21', 'CNS(=O)(=O)c1ccc(C(=O)[C@@H](C#N)C(=O)CC(C)(C)CC)cc1', 'O[C@@H]1c2cc(NC(=O)COCc3ccccc3)ccc2O[C@@H]1C(=O)N1CCCC1', 'C[C@@H](O)c1cccc(CS[C@@H]2CCCOC2)c1', 'NS(=O)(=O)Cc1nc(Oc2ccc(Cl)cc2)c2c(-c3cccc(Cl)c3)ccc([O-])c2n1', 'N#Cc1c(/C=C\\C(=O)c2cccc(F)c2)cn[nH]/c1=N\\OCCc1ccccc1', 'Clc1ccccc1NS[C@H]1CCCN(CCc2cccnc2)C1', 'Nc1c(/C=N/c2ccc(Cl)cc2)cc(Br)nc1CO', 'O=C(C1CC=C(c2ccc(Br)cc2)SC1)N1CCN(C(=O)COC(F)(F)F)CC1', 'OCS/C=C/c1cnc(C2CCCCC2)nc1Cl', 'ClC[C@@H]1CCC[C@@H]1COCCn1cccn1', 'Oc1cc(Br)cnc1OCc1ccnnc1Cl', 'NS(=O)(=O)/C(CNC(=O)c1ccc(Cl)cc1)c1ccc(C(=O)[O-])cc1', 'Clc1cc(F)cc(N[C@@H]2CCCC[C@@H]2O)c1', 'ClCc1cc(O)cc(F)c1I', 'Clc1cc(NSc2ccc(F)c([N+](=O)[O-])c2)ncn1', 'Clc1cccc2[nH]nc(SC[C@@H]3CCCO3)c12', 'Clc1ccc(Br)c(/N=N/Sc2nnc(N3CCOCC3)s2)c1', 'Clc1nnnn1/C([O-])=C/c1ccc2ccccc2c1', 'Oc1ccc([N+](=O)[O-])c(Cl)c1OC[C@H]1CCCO1', 'CCCC[C@@H](CC)CNC(=O)N[C@H]1c2ccccc2C[C@H]1O', 'Clc1ccccc1C(C(F)(F)F)[C@@H]1CCN(c2ccccc2)C1=O', 'O=C(c1cc(Cl)c2cccnc2n1)N1CC[C@@H](c2ccccn2)C1', 'Clc1ccc(C[NH2+]c2ccnc(-c3ccco3)n2)cc1', 'ClC(/N/N=C/c1ccccc1)C1CCOCC1', 'Clc1cc(Br)cc([C@@H]2CCCN2C(=O)c2cccnc2)n1', 'Nc1n[nH]c2c1[C@@H](c1ccccc1)CN(c1ccccc1Cl)C2', 'Clc1cc(Br)c(-n2ccnc2)c(Br)c1O', 'Clc1cccc2ncnc(N3CCOC4(CCCC4)C3)c12', 'Clc1nnnn1c1c(C(F)(F)F)ccc2c1OCO2', 'O=C([O-])/C=C/c1cccc(O[C@H]2CCN(c3ccccc3)C2=O)c1', 'Oc1cc(F)cc2c1NC(=O)N2Cc1cc(Cl)sc1Cl', 'O=C(CSCc1cc(=O)[nH]c(-c2ccccc2)n1)NC[C@H]1CCCO1', 'Clc1ccc(Cc2noc(C[NH2+]C3CC3)c2)cc1', 'Oc1cc(Br)c([O-])c(C#CCOc2ccccc2)c1', 'O=C([C@H](c1cccs1)N(c1cccc(Cl)c1)N1CCCC1)c1cccc(F)c1', 'Clc1nnc(CCC[C@H]2CCCO2)nc1C1CC1', 'Clc1ccccc1Oc1cnc2nnnn2c1[O-]', 'Oc1ccc(F)cc1CSC1=N[C@@H](c2ccc(F)c(F)c2)N1C1CCC1', 'Oc1cc(-n2nncc2F)nc(-c2ccccc2Cl)n1', 'NS(=O)(=O)Nc1cc(S(=O)(=O)C=O)ccc1C(F)(F)F', 'Clc1ccc([C@@H]([O-])Nc2ccc(Br)cc2Cl)cc1', 'Clc1ccc(-c2csc3sc(-c4ccoc4)cc23)cc1O', 'NS(=O)(=O)/N=C/Cc1ccccc1Br', 'Clc1ccccc1Cc1cnc([C@@H]2CCO[C@H]2C(CC2CC2)c2ccccc2)c(Cl)c1', 'Clc1cc(NSc2nnnn2Cc2ccccc2)cs1', 'O=[N+]([O-])c1cccc2c1CCN(C[C@H](O)CN1CCc3cc(F)cc(F)c3C1)C2', 'ClCc1csc(S(=O)(=O)N(Cc2ccccc2)[C@H](CO)c2ccccc2Cl)c1', 'Clc1ccc(Br)cc1-n1ncc(Br)c1C(=O)[O-]', 'O=C(Cc1nc2ncccn2c1C[NH2+]C1CC1)NC1CCCCC1', 'Clc1cc(Br)c2nc(-c3cc(Cl)cs3)oc2c1', 'Clc1cc(Br)cn1NC1CCCCC1', 'Clc1cccc(S[C@H]2CCC[C@@H](OCC(F)(F)F)C2)c1', 'OC[C@@H]1[C@H](OCc2ccccc2)c2ccc(F)cc2O1', 'O=[N+]([O-])c1cc(C2CCCC2)nc2nnnn12', 'Clc1cc(COC[C@H]2CCCN(c3ccccc3F)C2)sc1NC1CC1', 'OC[C@@H](SCc1cccc(C(F)(F)F)c1)N1CCN(C2CCN(C(=O)c3ccccc3)CC2)C1', 'Oc1ccc(/C=C2/C(=O)N(c3ccccc3)C(=O)N(c3ccc(Cl)cc3)c3ccc(F)cc32)cc1', 'NS(=O)(=O)/C(C#Cc1ccccc1)c1ccc(Cl)cc1', 'Clc1cc(/N=C/c2cnc(-c3cccs3)s2)ccc1NC[C@@H](O)c1ccccc1', 'NS(=O)(=O)N[C@H]1c2ccccc2C(=O)N1CC(N)=O', 'Clc1ccc(Br)cc1OS(=O)(=O)C[C@@H]1COc2ccccc2O1', 'Oc1ccc(/C=N/[C@@H]2CCCOC2)cc1', 'O=C([C@H](Nc1ncccc1Br)C1CC1)Nc1nccs1', 'Clc1ccccc1Oc1cc(Sc2nc3c(s2)CCCC3)cs1', 'Clc1cc(Br)cnc1N1CCO[C@@H](NC2CC2)C1', 'Clc1ccccc1-c1noc(CN2CC[NH2+]CC2)n1', 'Clc1cc(Cl)nn1/N/c1cccc([N+](=O)[O-])c1', 'Clc1ccccc1OC1CC[NH+](CC[C@@H]2CCN(C3CC3)C2)CC1', 'ClC[C@H](CS(=O)(=O)c1cc(F)ccc1F)Cn1cccn1', 'Oc1cc(OCOC2CCCC2)c(Br)c(Cl)c1OO', 'Oc1ccc(F)c(ON/C=[N+]([O-])c2ccc(Cl)cc2)c1', 'Nc1c(Cl)ncn2ccnc12', 'Nc1cn[nH]c(=O)c1N', 'ClC[C@H]1Oc2ccccc2NC1', 'ClC[C@@H]1Oc2ccccc2NC1', 'O=C(/C(c1cccc(F)c1)NCc1ccccc1)[C@H]1CC=CCC1', 'O=C(CS(=O)(=O)[C@H]2CCOc3ccccc32)N1CCc2ccccc21', 'CNc1nc(C(=O)N2CCC[C@H](CNc3ccc(OC)cc3)C2)c2ccccc2[nH+]1', 'Clc1ccc(C2(c3ccccc3F)C[NH2+]C2)cc1', 'Cc1nc(C(=O)N2CCC[C@@H]2C(=O)c2c(C)cc(C)cc2[N+](=O)[O-])c2ccccc2n1', 'Clc1ccc(Br)c(/N=[N+](\\[O-])c2ccc([N+](=O)[O-])cc2F)c1', 'O=[N+]([O-])c1cc(C(=O)NC[C@H]2CCC[NH2+]C2)[nH]c1N1CC=C(c2ccc(F)cc2)CC1', 'Clc1cccc(Br)c1O[C@H](C(=O)c1cccnc1)c1cccs1', 'Nc1n[nH]c(-c2cccc(Br)c2)c1[O-]', 'O=[N+]([O-])c1cc(OCCC(=O)NC2CC[NH+](Cc3ccccc3)CC2)ccc1Cl', 'O=C(c1ccsc1)N1CCN(S(=O)(=O)OC[C@@H]2CCCO2)CC1', 'O=C(c1cn2ccsc2n1)N1CCC(N2CCc3oncc3C2)CC1', 'Clc1cc(/C=N/Sc2cnn(-c3ccccc3)c2Br)ccc1[N+](=O)[O-]', 'ClC[C@@H]1CNC(=O)c2ccccc21', 'O=C(COc1ccc(Br)cc1)NCC[NH+]1CCCCC1', 'Clc1nnnc(-n2nc(-c3ccc(F)cc3)cc2Br)n1', 'O=[N+]([O-])c1cc([C@H]2CCCN(c3ncccn3)C2)c(F)c(Cl)c1', 'Clc1cccc2ncnc(NC[C@@H](O)c3ccoc3)c12', 'O=C(CCc1nnc(-c2ccccc2)o1)NCCC[NH+]1CCCC1', 'O=[N+]([O-])c1c(Cl)nnc2onc(-c3cccc([N+](=O)[O-])c3)c12', 'Clc1cc(NSc2nnc(COc3ccccc3Cl)[nH]2)cc(Cl)c1Cl', 'OC[C@@]1([C@H]2CCCN(C(=O)c3ccco3)C2)C(=O)C[C@@H]1Oc1ccccc1', 'Clc1ccsc1[C@H]1COc2ccccc21', 'NS(=O)(=O)C[C@@H](Cc1cn2ccccc2c1)c1ccccc1', 'Oc1cccc2ncnc(C(=O)Nc3cccc(C[S@](=O)c4ccccc4F)c3)c12', 'Oc1cccc(CC[NH2+]Cc2cccc(C(F)(F)F)c2)c1', 'O=C(c1cccnc1SCC(=O)Nc1cccc(Cl)c1)N[C@@H]1CCCC[C@@H]1CO', 'OC[C@@]1(NC2CC2)COc2ccccc21', 'Clc1cccc(NC2CCN(C(=O)C3=c4ccccc4=[NH+]C3)CC2)c1', 'CNS(=O)(=O)[C@@H]1CCN(C(=O)[C@H]2CCCN2C(=O)Cc2ccccc2)C1', 'N#C[C@@H](c1ccccc1)[C@H]1CCC[NH+](C2CN(C(=O)Cc3cnn(C4CC4)c3)C2)C1', 'ClCc1cn[nH]c1', 'Clc1ccc(Br)cc1[C@@H](CO)N[C@@H](CO)c1ccco1', 'Clc1ccc(-c2csc3[nH]cc(Br)c23)c(Br)c1', 'O=C(CSCc1ncc(Cl)cn1)NC[C@@H]1CCc2ccccc2O1', 'Cc1nccn1-c1ccc(NC(=O)N[C@H](C)c2c(C)nn(C)c2C)cc1O', 'O=C(CS(=O)(=O)c1cc(F)ccc1F)N[C@@H](CO)c1ccco1', 'Clc1ccsc1[C@H](O)CNC(=O)N[C@H]1CCCc2ccccc21', 'O=C(c1cn2ccccc2n1)N1CC[C@@H](CO)[C@H]1NC(=O)c1ccccc1Cl', 'ClCc1csc2nc[nH]c(=O)c12', 'Clc1cc(Br)c2nc(O)sc2c1', 'Clc1ccccc1Oc1cnn(C[C@@H]2CCCC[C@@H]2O)c1', 'OCc1nnsc1C(=O)c1ccc(Br)s1', 'NS(=O)(=O)CCNC(=O)[C@@H]1OCC(=O)N(C)[C@H]1c1ccc(Cl)cc1', 'Oc1ccc(F)c(F)c1C[NH+]1CCC(OCc2ccccc2)CC1', 'OCc1nnsc1[C@H]1CCCN1C(=O)c1ccccc1', 'O=C(CC[NH+]1CCC(O)(c2ccccc2)CC1)Nc1cccc2ncccc12', 'Clc1nnc(Br)cc1Oc1ncccn1', 'Clc1cccc(/N=N\\Cc2ccc[nH]2)c1F', 'N[C@@H](CCOc1cccc(C(F)(F)F)c1)N(C)Cc1ccc(-n2cncn2)cc1', 'Clc1ccsc1/C=N/NC1=NS(=O)(=O)c2ccccc21', 'O=C(c1ccsc1)c1c(O)ccnc1SCC(F)F', 'O=C(c1cc([S@](=O)Cc2cccs2)nc2ccccc12)c1ccc(F)cc1', 'Clc1ccc(Br)c([C@H]2c3ccccc3C=CN2C(=O)c2ccc(O)cc2)c1', 'Clc1nncnc1N=C\\c1ccc(Cl)cc1Cl', 'Clc1ccc(C2=CSCCN2)cc1[N+](=O)[O-]', 'CC[C@@H](OC(=O)CN1C(=O)/C(=C/c2ccc(Oc3ccccc3)cc2C)SC1=S)C(=O)[O-]', 'CC(C)(C)C(=O)Nc1c(NCc2[nH+]ccn2C2CCCCC2)cnn1-c1ccc(Cl)cc1', 'O=C(c1c[nH]cn1)[C@H]1CCCN1C(=O)Cc1ccco1', 'Cc1ncsc1[C@@H](O)N1CCN(Cc2cccc(F)c2)CC1', 'O=C(c1ccsc1)N(Cc1cccs1)[C@@H]1CCS(=O)(=O)C1', 'O=C(/N=C/CCOCc1ccccc1)NC1CC[NH+](Cc2ccccc2)CC1', 'Clc1ccc(C[NH2+]c2cccnc2)cc1', 'Clc1ccc(-c2nnnn2[C@@H]2C[C@@H]2CO)cc1', 'O=C(/C=C/c1c(Cl)ccc2ccccc12)N1CCCC[C@@H]1C[C@@H](O)CO', 'N[C@@H](C(=O)Nc1ccccc1C(F)(F)F)[C@H]1CCCO1', 'NS(=O)(=O)C[C@H]1C[C@@H]1c1ccccc1', 'Clc1cccc(Br)c1NC[C@H]1CC[C@H](c2ccccc2)[NH2+]1', 'Clc1ccsc1SCC(=O)N1CCC[C@H]1c1ccc2c(c1)OCO2', 'Clc1nncc(CN2CCC[C@@](O)(CCn3cnc4ccccc4c3=O)C2)c1', 'Clc1ccccc1Cc1nccc2ccc(C(=O)Nc3cnn([C@@H]4CCC[NH+](C)C4)c3)cc12', 'Nc1c(C(=O)N2CC[C@@H](c3ccco3)C2)cnn1C(C)C', 'ClCc1csc(-c2ncc(Br)s2)n1', 'O=[N+]([O-])c1c([O-])[nH]c(=O)c2ccc(F)cc12', 'Clc1ccccc1[C@H](Cl)[C@H]1COc2ccccc21', 'O=[N+]([O-])c1c([N+](=O)[O-])cnc2cc[nH]c12', 'Oc1ccc([N+](=O)c2cc(COC3CCCC3)c[nH]2)cc1', 'ClCc1n[nH]c2cc(-c3cc(F)cc(F)c3F)oc12', 'ClC[C@H]1OCc2ccccc21', 'COc1nn[nH]c1C(C)(C)OC(=O)Cn1c(=O)n(-c2ccc(C)c(C)c2)c(C)cc1=O', 'Clc1ccc(CO)n1[C@@H]1CC(=O)N(Cc2ccco2)C1=O', 'Clc1cc(Br)c2nc(OCc3cscn3)[nH]c(=O)c2c1', 'NS(=O)(=O)Cc1cn2ccsc2n1', 'Clc1ccc(-c2csc3[nH]c(NC[C@@H]4CCCO4)cc23)cc1', 'Clc1ccccc1OCCC[NH+]1CCCCC1', 'COc1nc(N)cc(C(=O)N2CCC[C@@H](CO)C2)c1N', 'O=C(Cc1noc(C2CCCCC2)n1)N[C@@H]1CCSc2ccccc21', 'Clc1ccccc1C(C1CC1)[C@H]1Sc2ccc(Cl)cc2N1c1ccccc1', 'O=C([O-])/C(CNC(=O)c1cc(-c2cccs2)on1)c1ccccc1', 'Oc1cc(C=N[C@@H]2CC[NH+]=C2c2ccco2)ccc1O', 'O=C([O-])c1nc(CSc2cc(F)ccc2F)c(F)cc1F', 'Clc1ccc(Br)cc1/N=N\\Nc1cccc(C(=O)[O-])c1', 'Oc1cccc(Nc2nnc(SCc3nc4cccc(F)c4s3)n2N2CCOCC2)c1', 'NS(=O)(=O)Cc1ncco1', 'Clc1cc(F)cc(CN2CC[NH+](CCOc3ccccc3)CC2)c1', 'Clc1cc(F)ccc1[C@H](O)c1nncn1C1CC1', 'Clc1ccccc1NS[C@H](C(=O)[O-])C(=O)Nc1ccccc1', 'Clc1ccc(-c2csc3s/cnc23)nc1', 'Clc1ccc(-c2csc3s/cnc23)nc1', 'O=[N+]([O-])c1ccncc1Sc1ncc[nH]1', 'Oc1ccc(Oc2nc[nH]n2)c2ncccc12', 'Oc1ccc([C@H](c2cc3ccccc3[nH]c2=O)c2cc[nH]n2)cc1', 'CCCCN(Cc1cccc(C#N)c1)C(=O)N[C@@H]1C[C@@H]1C', 'C[C@@H](c1ccncc1)C(=O)N1CCCC(Cn2cc(Cl)cn2)C1', 'Clc1cccc2cc([C@H](O)CSc3nc4ccccc4s3)oc12', 'O=[N+]([O-])c1c(Cl)n2nnc(/N=C/c3ccccc3)c2nc1O', 'Clc1ccc(-c2cc(SC[C@@H](CCl)N3CCOCC3)ncn2)cc1', 'Cc1ncsc1C(=O)N1CCC[C@@H](Oc2nc3ccccc3cc2CO)C1', 'ClC[C@@H]1Oc2ccccc2C(=O)N1c1ccccc1C(F)(F)F', 'Clc1ccccc1N1NC[C@@H](Cc2cnn(-c3ccccc3)n2)C1', 'Clc1cccc(NC(=S)N2CC[NH+](Cc3ccccc3)CC2)c1', 'COc1nn[nH]c1/C=C/C(=O)N[C@@H](CO)c1ccccc1', 'Clc1nncc(Nc2cccc(C(=O)[O-])c2)n1', 'Clc1ccc(-c2n[nH]c3c2[nH]c(=S)n3C(F)(F)F)cc1', 'CNC(=O)CCC(=O)NC[C@@H]1CCCC[C@@H]1C', 'Clc1cccc([C@@H](SCc2ccccc2)C(F)(F)F)n1', 'Oc1cccc(Br)c1OC[C@@H]1CN2CCC[C@@H]2CO1', 'O=C(/C(c1cccs1)=[NH+]/Nc1cccc(C(F)(F)F)c1)N1CCN(c2cccc(Cl)c2)CC1', 'Clc1cc(CN(c2c(Cl)cccc2Cl)SC2CCCC2)c2ncccn12', 'O=C(c1c[nH]nc1CNc1ccc(-c2[nH+]c[nH]n2)cc1)N1CCOCC1', 'ClC(c1cc2ccccc2oc1=O)c1cccnc1Sc1ccc(F)cc1', 'O=C(c1c[nH]cn1)c1cccc(CN2CCC[C@H](O)C2)c1', 'Cc1ccc(S(=O)(=O)N2CCCC[C@@H]2C(=O)NOc2cncc(C(F)(F)F)n2)cc1', 'Clc1ccc(Br)cc1/C=C(/C#N)C(=O)N[C@@H]1CCS(=O)(=O)C1', 'OCc1n[nH]cc1[C@H](CCO)c1ccc(Cl)cc1', 'O=C(c1ccccc1)c1ccc(Br)cc1OCC[NH+]1CCCC1', 'O=C(/C(c1cccs1)c1ccc(Br)cc1)c1ncc[nH]1', 'Oc1ccc([N+](=O)c2cc(C#N)cc(Cl)c2)cc1O', 'O=C(Cc1nn[nH]n1)Nc1ccc(C(=O)N2C[C@@H](C)O[C@H](CCc3ccccc3)C2)cc1', 'CCNC(=O)[C@H]1C[C@H](c2ccccc2F)N(C(=O)c2ccc(F)c(F)c2F)C1', 'Clc1ccc(COc2nc3cccc(Cl)c3[n-]2)cc1', 'Clc1ccsc1Nc1ncnc(CO)c1-n1cccc1', 'Oc1ccccc1[C@H](N=C(CO)c1c[nH]c2ccccc12)c1cccs1', 'Clc1ccc([C@@H](Br)C(F)(F)F)c(F)c1', 'Nc1[nH]ncc1COC(=O)[C@H]1C[C@H]1c1ccc(Cl)cc1', 'Clc1cccc(F)c1N(C[C@@H]1Oc2ccccc2OC1=O)c1ccccc1', 'O=[N+]([O-])c1cc(Br)c(O)c(F)c1C(F)(F)F', 'Clc1ccc(Br)c(CCC(=O)N2[C@H](c3ccccc3)OC2(C)O)c1', 'Clc1ccc(Br)cc1S[C@H]1CCC(=O)NC1=O', 'O=[N+]([O-])c1c([O-])c(COc2cccnc2)c2ccccc2[n+]1[O-]', 'Clc1cc(F)cc(N/N=C2\\N=NC3=C2CCc2ccccc23)c1', 'Clc1nnnc(Sc2ccnc(N3CCOCC3)c2)n1', 'Clc1ccccc1[C@H]([NH3+])c1cc(-c2ccc(Cl)cc2)nn1CCO', 'O=[N+]([O-])c1c(Cl)nsc1C(=O)N1CC[C@@H](Oc2ccc(F)cc2)C1', 'Clc1cc(NC2=NC=CC2)ccc1O', 'O=[N+]([O-])c1c(Cl)nn(-c2ccc([N+](=O)[O-])cc2)c1[O-]', 'Clc1nnnn1/N=C/c1cc(Cl)ccc1O', 'Clc1nncnc1N[C@H]1CCc2ccccc21', 'O=C(CS(=O)(=O)[C@H](Cc1ccccc1)N1C(=O)c2ccccc2C1=O)c1ccco1', 'Clc1ccc(-c2cc[nH]n2)c(OCCC(=O)[O-])c1', 'O=[N+]([O-])c1c([O-])c2nc(-c3ccccc3)n[nH]c2c1O', 'Clc1nnn(-c2ncccc2[N+](=O)[O-])c1I', 'O=[N+]([O-])c1c(OCCO[C@@H]2CCO[C@@H]2c2ccccc2)c2ccccc2o1', 'Oc1cc(F)ccc1[C@@]1(COc2ccccc2F)CCO1', 'CC(C)=CC(=O)C[C@@H]1C(=O)NCCN1C(=O)c1cccc(Cl)c1', 'N=C1CCCC1=CCc1c(O)cccc1Cl', 'N#CC(/N=C/c1ccc([N+](=O)[O-])cc1)=C/C(=O)Nc1nccs1', 'O=C(c1c[nH]nc1C1CC1)N1CCC[C@H](N2CCN(C3(c4ccccc4)CCO3)CC2)C1', 'O=[N+]([O-])c1c(F)cc2nc(N3CCC(O)CC3)n[nH]c12', 'Clc1ccsc1[C@@H](O)c1ccncc1', 'O=C(C[NH+]1CC[C@@H]2[C@@H](c3ccccc3)[C@H]2C1=O)NCc1ccccc1', 'O=C(CNS(=S)(=O)c1ccccc1Cl)NCC(F)(F)F', 'Oc1cccc(SCn2nc3cccnc3n2)c1', 'OCc1nnsc1NC[C@H](O)c1ccccc1F', 'N=CCc1nnc(C(=O)N(C)Cc2ccccc2OCC(F)F)s1', 'Clc1ccccc1OCc1ccn([C@@H]2CCSC2)c1', 'Oc1ccc(O[C@H](CNC(=O)c2ccco2)N2CCSCC2)cc1', 'Oc1cccc(S[C@@H](c2ccccc2F)N2CCC(c3c(F)cccc3F)CC2)c1', 'Oc1ccc(F)cc1N[C@@H]1CCCSC1=O', 'N=CCNS(=O)(=O)c1cccc(C(=O)N[C@@H]2CCCC[C@@H]2C)c1', 'Clc1ccccc1Oc1cc(SCc2csc(C(F)(F)F)n2)[nH]n1', 'Clc1cc(/C=Cc2[nH+]cn(-c3ccccc3)n2)ccc1NN1CCCC1', 'Clc1ccccc1-c1csc([C@@H]2C(=O)NC(=O)c3ccc(Cl)cc32)n1', 'Clc1ccc(-c2n[nH]cc2N2CCCC2)nc1[N+](=O)[O-]', 'O=C(c1cn2ccsc2c1NS(=O)(=O)c1ccccc1)N1CCC[C@@H]1c1cccc(Cl)c1', 'ClCc1cncc(C(=O)OCc2nc[nH]n2)c1', 'COC(=O)c1cnn(Sc2nncn2C2CC2)c1C', 'Clc1nnn(-c2cnn(Cc3cccnc3)c2Cl)c1Cl', 'Clc1cccc(C[NH2+]Cc2cccc3c2OCCO3)c1', 'NS(=O)(=O)C[C@H](NC1=NS(=O)(=O)c2ccccc21)c1ccccc1', 'Nc1cnnn1Sc1ccccc1', 'Clc1ccccc1NS[C@H](Nc1cccnc1Cl)c1ccccc1', 'O=[N+]([O-])c1c(F)cc(Br)cc1NC[C@H](O)C1CCC1', 'O[C@@H]1c2cc(-c3ccco3)ccc2O[C@@H]1c1cccc(Br)c1', 'Clc1ccc(Br)cc1O[C@H](CO)C(F)(F)F', 'Clc1cccc(-c2n[nH]cc2C2=NN=C(Sc3nccn3Cc3ccccc3)C2)c1', 'Oc1cc(F)ccc1[C@@H]1CCCCN1C(=O)c1cc([N+](=O)[O-])cs1', 'Clc1cc(NSc2ccc(Cl)s2)ccc1C(F)(F)F', 'N#CC1(SCc2ccco2)CCCCC1', 'Clc1cc(/N=C/c2cccc([N+](=O)[O-])c2)sc1O', 'Oc1cccc2cc(O[C@H]3CCOC3)cc(C(F)(F)F)c12', 'N=Cc1nonc1-c1ccc(OC(F)F)c(C)c1', 'Oc1ccc(O[C@@H]2C[C@@H](C(=O)[O-])[C@H]2C(=O)Nc2ccccc2)cc1', 'Oc1cc(F)c([C@H](O)C(F)(F)F)cc1', 'Clc1ccccc1Cn1c[nH+]c(-c2cccc(Cl)c2)n1', 'Oc1cc(F)nc(NC[C@H]2CCCO2)c1', 'ClC[C@@H]1Oc2ccccc2[C@@H]1Cc1c(F)cccc1Cl', 'Oc1ccc([N+](=O)[O-])c(/C(=N/c2ccc(Cl)cc2)[n+]2ccccc2Cl)c1', 'Oc1ccc(F)c(C/C(Cl)=C2\\CCCCN2)c1', 'C=C[C@@H](Sc1cccc(F)c1)C(=O)N/N=C/c1ccc(C)cc1', 'Oc1ccc(/C=N\\CCN2CCN(S(=O)(=O)c3ccc4c(c3)OCO4)C[C@H]2c2ccccc2)cc1', 'O[C@@H]([C@@H](Cl)CO)C(=O)OCc1ccccn1', 'Clc1cccc([S@@]C2SCCCC2)c1', 'O=[N+]([O-])c1cc(Br)c(Br)c(C(F)(F)F)c1O', 'Clc1cccc(Br)c1NC1CC[NH+](CCN2CCCC2)CC1', 'Nc1[nH]ncc1C(=O)O[C@@H]1CCCC[C@H]1c1ccccc1', 'CCCCO[C@H]1C[C@H](O)CN1Cc1cccc(OC)c1', 'Clc1cc(/N=C\\c2ccc([N+](=O)[O-])cc2F)cs1', 'Clc1ccc(CN2N=C3SCC=C3SC2)cc1N1CCCC1', 'O=[N+]([O-])c1c(Cl)nnc2[nH]nc(C3CCCCC3)c12', 'Nc1[nH]ncc1S(=O)(=O)CCC(F)F', 'Clc1nncc(Cl)n1', 'Clc1nncc(Cl)n1', 'Clc1nncc(Cl)n1', 'Oc1ccc(F)cc1S(=O)(=O)N(C1CC1)[C@@H]1CCS(=O)(=O)C1', 'C=CCC[C@@H](C)OC(=O)c1ccc(-c2nc(C)c3ccccc3n2)s1', 'Clc1cccc([S@@](=O)/C=C/c2ccc([N+](=O)[O-])cc2)c1', 'ClC[C@H](Br)c1c(Cl)cccc1Cl', 'Oc1cccc(CCNc2c([N+](=O)[O-])c(O[C@@H](C)c3ccccc3)no2)c1', 'OCc1n[nH]c(Sc2nc(-c3ccccc3)no2)c1[N+](=O)[O-]', 'Clc1nnnc(O[C@@H]2CCCN(c3ccccc3)C2=O)c1Br', 'CNC(=O)N1C[C@@H](c2ccccc2F)OC1=O', 'Clc1ccccc1-c1csc(CN[C@@H]2CCC[C@@H](C(F)(F)F)C2)n1', 'COC(=O)C1(C#N)CC[C@H](n2ccc(=O)[nH]c2=O)CC1', 'Oc1ccc(Oc2nc(=O)c3cccnc3n2N)c([N+](=O)[O-])c1Cl', 'Clc1cc(NSc2nc(-c3cccs3)nc3c2CCCC3)ccn1', 'Clc1ccc([C@@H](CSCC(F)(F)F)n2c(CO)nc3ccccc32)cc1', 'OCc1nc(C(=O)NCc2[nH]ccc2Cl)n[nH]1', 'Clc1ccccc1OCc1cc(=O)n2c(n1)SCCN2', 'Clc1nnnc(S(=O)(=O)N2CCOCC2)c1-n1cccc1', 'O=[N+]([O-])c1ccnc2ccc(F)c(-c3ncoc3C(F)(F)F)c12', 'COCC[C@@]1(Cc2ccc(F)cc2)NCCn2cccc21', 'Oc1ccc([N+](=O)[O-])c(N[C@@H]2CC(=O)c3ccc(Cl)cc32)c1', 'N[C@@H]1[C@H](c2cccc(F)c2)CCN(Cc2cncc(-c3cc(OC)c(OC)cc3)n2)C1', 'O=C([C@H](SC1=NC=NC1=O)c1ccccc1)c1ccc(Br)cc1', 'O=C(c1cnc(N2CCc3ccccc32)nc1)N1CC[C@@H]([NH+]2CCCC2)C1', 'Clc1cccc(/N=C(\\[O-])NCc2ccco2)c1O', 'CC[C@@H](OC(=O)c1cccc([N+](=O)[O-])c1)[C@H]1CCCO1', 'Clc1cccc2c1C/C(=C\\c1ccccc1O)[C@@H]2c1ccccc1F', 'Oc1cc(Br)c2sccc2c1Oc1c(F)cc(F)c(F)c1F', 'CC[C@@H](C)OC(=O)c1nc(C)cc(C(N)=O)n1', 'Clc1ccc(-c2csc3[nH]nnc23)cc1O', 'O=[N+]([O-])c1cc(C(=O)O[C@H]2CCN(c3ncnc4sccc34)C2)cc([N+](=O)[O-])c1', 'Clc1ccsc1Nc1nc(-c2ccc(Br)s2)no1', 'Clc1ccccc1OC1CC[NH+](Cc2cccc(Cl)c2)CC1', 'ClCc1ncn[nH]1', 'O[C@@H]1Cc2ccccc2[C@@H]1Cc1c(F)cccc1F', 'Oc1cc(-n2ncc(Cl)c2Cl)ccn1', 'Clc1cccc(Br)c1C[NH2+][C@H]1CCC[C@H](Cc2ccccc2)C1', 'Oc1cc(-n2nnnc2SSc2ccc(Cl)cc2[N+](=O)[O-])cc(Br)c1F', 'NS(=O)(=O)NC[C@@H](c1ccco1)N1CCN(C(=O)[C@@H]2CCc3ccccc3C2)CC1', 'ClCc1csc(C[C@@H]2CCCOC2)n1', 'Oc1ccc([N+](=O)[O-])c(CN2C[C@@H](O)C[C@H]2c2ccccc2)c1', 'OC[C@@H](Cc1cc(F)cc(Br)c1)OCC(F)(F)F', 'ClC[C@H]1COc2ccc(F)cc21', 'ClC[C@@H]1COc2ccc(F)cc21', 'O=C(CS(=O)(=O)[N-]c1cc(F)ccc1SCc1cccs1)Nc1ccccc1', 'CNC(=O)c1cc([N+](=O)[O-])c(N2CCCC2)c2c1cnn2C[C@@H](C)CC', 'Clc1ccc(-c2cccs2)nc1[N-]S(=O)(=O)c1ccc(Cl)cc1', 'Clc1ccccc1OCC[NH2+][C@@H]1CCc2ccccc21', 'Clc1nnc([S-])n1CCc1ccccc1C(F)(F)F', 'O=C(c1cnc(SCc2ccc(F)cc2)s1)N1CC[C@H]2CCCC[C@@H]21', 'Clc1ccc(-c2n[nH]c(C[NH+]3CCC(COc4ccc(F)c(F)c4)CC3)n2)cc1', 'NS(=O)(=O)NCc1csc(C(=O)N2CCCC[C@@H]2C)n1', 'Clc1ccsc1[C@@H](Cl)Cc1cc(Cl)ccc1F', 'Oc1ccccc1[C@@H]1CCCN1S(=O)(=O)/C=C/c1ccco1', 'Clc1ccc(C(N)=[NH2+])cc1F', 'Oc1ccc([N+](=O)c2ccc(C(=O)[O-])cc2F)cc1', 'Oc1cc(-n2cc(/C=C3/C(=N/c4ccccc4)c4ccccc4S3)C2=O)ccc1O', 'ClCc1nnsc1NCc1ccnn1C1CCOCC1', 'Clc1ccc(Br)cc1[C@@H](O)COC(F)(F)F', 'O=[N+]([O-])c1c(OCOC[C@H]2CCCO2)c(Cl)ccc1F', 'O=[N+]([O-])c1c([N+](=O)[O-])cnc2[nH]cnc12', 'Oc1ccc(Oc2ncccc2C2(C[NH2+]C(F)(F)F)CC2)cc1', 'O=[N+]([O-])c1cccc2c1[C@H](CO)C2', 'Clc1nncc(CSc2nccnc2C2CC2)n1', 'Clc1ccsc1CN[C@@H]1CCC(=O)N(C2CCOCC2)C1', 'O=C(CNc1cnc2ccccc2n1)NCCn1cc[nH+]c1', 'O=C(c1cnn(C2CCCC2)c1)N1CC[C@H](C(=O)[O-])c2ccccc21', 'Clc1cccc(-c2nnc(NC[C@@H]3CCCO3)[nH]2)c1', 'Clc1ccsc1S(=O)(=O)N1CCc2nccnc2C1', 'O=C(c1c[nH]c2nc[nH]c12)N1CCC[C@H]1C[C@H](O)c1ccccc1', 'Clc1nnc([S-])=C(/C(=O)OCCN2CCOCC2)c1Cl', 'Clc1cc(/C=Cc2cccc[n+]2[O-])cn1C1CC1', 'O=[N+]([O-])c1c(Cl)nc2[nH]ncc2c1Cl', 'O=C(/C=C[C@@H]1CCCS1(=O)=O)c1cccc2ccccc12', 'Clc1cccc(F)c1CCNC[C@]1(O)CCSC1', 'Clc1ccsc1/N=N/c1ccc(Cl)cc1Br', 'Clc1ccc(Br)cc1-c1cc2c(cc1Cl)NC[C@@H](c1ccc(F)cc1)N2', 'Clc1ccccc1Oc1cnc2n1C[C@@H](OCc1cccnc1)C2', 'COc1nnnc(N[C@H](C)c2ccc(F)c(F)c2)c1Cl', 'O=C(/C(c1ccccc1)[C@@H]1CC=CCC1)c1ccc([N+](=O)[O-])cc1', 'Clc1ccc([C@@H](Br)[C@H](Cc2ccccc2)[NH+](C)C2CC2)cc1', 'Clc1ccsc1-c1nonc1NCc1ncc[nH]1', 'Clc1ccc(-c2cccnc2N=[N+]2CCCC2)cc1', 'Oc1ccc(/C=N/[NH+]C(=O)COc2ccccc2Br)cc1Br', 'Oc1cc(/C=N/NC(=O)C[C@@H](c2cccs2)N2CCOCC2)nc2ccccc12', 'O=C(c1cnnn1-c1ccccc1)N1CCC[C@H](c2ncccc2Br)C1', 'O=C(COc1cc(F)ccc1Br)N[C@@H]1CCSS1', 'Clc1cccc(C(=O)OC2CC[NH+](Cc3ccccc3Cl)CC2)c1', 'Clc1cc(/N=C\\Cc2ccccc2Br)cs1', 'Clc1nnn(-c2ncn[nH]2)c1CNC(=O)c1ccc[nH]1', 'Clc1ccc(Br)c(F)c1CN[C@@H](c1nccs1)c1ccccn1', 'O=[N+]([O-])c1c(-c2c[nH]nn2)cnnc1NCc1ccc(OC(F)(F)F)cc1Cl', 'C[C@@H](NCC1CCC(C(N)=O)CC1)c1nccnc1C1CC1', 'Oc1cc(-n2cn/cc2)ccc1N[C@@H]1CCC[C@H](C2CC2)C1', 'Clc1cc(/N=C\\c2sccc2CO)sc1NCc1ccc(Cl)cc1', 'OC[C@@H](Sc1nnc(-c2cccc(F)c2)n1C1CC1)c1ccc2cc[nH]c2c1', 'O=C(CNc1nn2nc(-c3cccc(F)c3)oc2c1Cl)Nc1cc([N+](=O)[O-])ccc1F', 'Clc1ccc(Br)cc1SC[C@H]1Cc2ncccc2-c2ccccc21', 'Clc1ccccc1CC(C[C@@H](O)CO)NNC(=O)c1ccsc1', 'COc1nn(CC)c(CN2CC(=O)N(c3ccc(C)cc3C)C[C@@H]2c2ccccc2)c1C', 'Nc1nonc1C[NH2+]C1CCN(CC(=O)c2ccccc2)CC1', 'N=Cc1nonc1Cc1cc(C)ccc1OCC(F)(F)F', 'Clc1ccc(COC[C@H]([NH2+]Cc2ncc[nH]2)c2ccccc2)cc1', 'Oc1cc(Br)c2[nH]nc(C(=O)/C=C/c3nnnn3-c3ccccc3)c2n1', 'O=C(CS(=O)(=O)[C@H]1CCCc2ccccc21)N/N=C/c1cnc2ccccc2c1', 'Clc1ccc(-c2cc[nH]n2)nc1[C@@H]1CCCN(C(=O)Nc2ccccc2)C1', 'Clc1ccc([C@@H](N[C@@H](C#N)c2ccccc2)C(F)(F)F)cc1', 'COc1nn[nH]c1C(=O)NC[C@@H](C)c1cccc(C)c1', 'Clc1ccc([C@@H]([O-])NCc2cc(Cl)ccc2[N+](=O)[O-])cn1', 'Cc1nnnn1CC(=O)N[C@@H](C)c1nc(C)cs1', 'O=C([O-])[C@@H]1CSCN1C(=O)Cc1ccccc1Cl', 'Clc1nnnn1/N=C\\c1cc([N+](=O)[O-])ccc1O', 'Oc1cccc(NC2=NC(=O)C3=Cc4ccccc4C(c4ccccc4)N32)c1', 'Clc1ccc(-c2cc[nH]n2)nn1', 'Clc1ccc(-c2cc[nH]n2)nn1', 'Clc1ccccc1OCC[NH+]1CCN(c2nc3ccccc3s2)CC1', 'O=C([C@H]1COCCN1S(=O)(=O)c1ccccc1Cl)N1CCC(c2nccs2)CC1', 'O=[N+]([O-])c1cc(Br)c2nccc(=O)n2c1N', 'ClCc1csc2nc(C(F)(F)F)cn12', 'NS(=O)(=O)N1C(=O)/C(=N/c2nc(-c3ccccc3Br)n[nH]2)c2ccccc21', 'O=C(c1cn2ccsc2n1)N1CC[C@](N2CCOCC2)C1', 'Clc1ccccc1N1N(C[C@@H]2CCCO2)C=C1c1cccc(Cl)c1', 'COc1ncc2nn(C)c(=O)c(C=O)c2c1C', 'C[C@@H](CN1CCCCC1=O)[NH2+]CC(=O)Nc1ccccc1C(F)(F)F', 'ClC[C@@H]1CN(c2ncccn2)C[C@H]1c1ccccc1F', 'Clc1ccc(Br)cc1S(=O)(c1ccc(F)cc1)C(=O)Nc1ccc(Br)cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1[C@H]1Cc2ccccc2O1', 'Clc1ccccc1OCCN1CC[NH+](CC2CC2)CC1', 'Nc1[nH]nc(C2CC2)c1Br', 'O=C(c1cccc(Cl)c1)N1CCC[C@]2(CCN(S(=O)(=O)c3ccccc3)C2)C1', 'Oc1ccc(F)cc1[C@H]1CCCN(c2nccs2)C1=O', 'Oc1ccccc1OSC(=S)N1N=C(c2ccc(Br)cc2)C(=O)[C@H]1c1ccccc1O', 'Clc1ccccc1C[NH+]1CCC(CNc2ccc(Cl)cc2[N+](=O)[O-])CC1', 'O=[N+]([O-])c1ccnc2c1[C@H](NCc1ccco1)C2', 'ClCc1cc(Br)c(Br)s1', 'O=C(CSC(/[O-])c1ccc(Cl)c(Cl)c1)NNc1ccccc1Cl', 'ClC[C@@H]1C[C@@H]1c1cccc(Br)c1', 'NS(=O)(=O)/N=C(\\[O-])CCS(=O)(=O)c1ccccc1N(C)C', 'N=CCn1ncnc1SCC(N)=O', 'Clc1ccc(-c2cc([C@H](C#N)C3CC3)cs2)cc1', 'Cc1sn(-c2ccc(F)cc2)c(=S)c1/C=N/NC(N)=O', 'ClCc1cn[nH]c1N1CCC[C@H]1c1ccccn1', 'OC[C@@H]([NH2+]Cc1ccncc1)Cc1ccccc1', 'O=C(/C(CN1CCN(c2ccccc2)CC1)c1ccccc1)S[C@H]1CCS(=O)(=O)C1', 'O=C(CCS(N)=C(Cl)Cc1ccccc1)NC[C@@H](O)c1ccc(F)cc1', 'COc1nnn(-c2ccc(NC(=O)[C@H](C#N)c3cccnc3)cc2)c1', 'COC[C@@H]1OC(=O)c2cc(Br)ccc21', 'COc1nn(-c2c(NC(=O)c3ccccc3)c3ccccc23)cnn1C', 'CNc1[nH+]cccc1C(=O)N1CC[C@@H](c2ccccc2)C1', 'O=[N+]([O-])c1ccncc1S[C@@H]1CC(=O)N(c2ccccc2F)C1', 'OC[C@@]1(c2ccccc2)N[C@H](c2cccc(F)c2)C(=O)N1COc1ccccc1', 'OC1=Nc2ccccc2C(c2ccccc2)(c2ccc(Br)cc2)C1', 'Clc1nncc(OCCCO[C@H]2COc3ccccc3O2)c1[N+](=O)[O-]', 'C=CCC[S@@](=O)Cc1nc2ccccc2n1C', 'Oc1ccc([N+](=O)c2cc(Cl)ccc2O)c2ncnn12', 'Clc1ccc(Br)cc1[C@@H]([NH2+]Cc1ccccc1)C1CC1', 'Clc1nncc(F)c1C(F)(F)F', 'Clc1ccc(C(=O)O[C@@H]2C[C@@H](c3ccccc3)[NH+](Cc3ccccc3F)C2)cc1', 'Oc1ccc(F)cc1O[C@@H](CO)C(F)(F)F', 'Clc1cc(Cl)nnc1O/C=C(/CC#N)c1ccc(Cl)cc1', 'Oc1cccc2ncn([C@@H]3CCCC[C@@H]3O)c12', 'O=C(CC1=CCCCC1)NNC(=O)[C@@H]1CC(=O)N(c2ccc(C#N)cc2)C1', 'Clc1cc(Cl)cc2c1NN(c1ccccc1C(F)(F)F)NC2', 'O=[N+]([O-])c1ccc(F)cc1CNC(=O)N1[C@@H](C(=O)[O-])CCN1c1ccccc1', 'O=C(CS(=O)(=O)[N-]C[C@@H](O)c1ccccc1)Nc1ccc(F)cc1', 'Oc1ccc(O[C@@H]2C(=O)Nc3ccccc32)c(Cl)c1Cl', 'Cc1snnc1Sc1nc(N)c2c(C)cccc2n1', 'Oc1ccccc1Cn1ccnc1Sc1ncnc([O-])c1', 'ClCc1cc([N+]([O-])=Nc2ccccc2Cl)oc1N1CCCC1', 'OC[C@H](O)c1ccc(C(F)F)cc1Br', 'ClCc1cn[nH]c1n1cccn1', 'N#C[C@@H]1Cc2ccccc2N1C(=O)c1cccc(NC(=O)c2sc(C3CC3)nn2)c1', 'O=C(/C=C1CCCC[C@@H]1C(=O)NCc1coc(-c2cccs2)n1)c1ccccc1Cl', 'Oc1ccc(/C=N/Sc2nnc3c(-c4ccc(F)cc4)nnn3c2[O-])cc1', 'Clc1cccc(CC[NH2+]C2CCN(CCO)CC2)c1', 'Clc1ccc(-c2csc3[nH]ccc23)nc1O', 'O=[N+]([O-])c1ccnc2c1CCC[C@@H]2Nc1ccc(Cl)c(F)c1', 'Clc1nnnc(Oc2cncc([N+](=O)[O-])c2)n1', 'C[NH+](C)Cc1ccc2ccccc2c1Cl', 'ClCc1nonc1[C@@H]1CCCN(c2ncccn2)C1', 'Clc1cc(Cl)nn1Cc1cnc2ccccc2c1[O-]', 'Clc1ccc(-c2nnnn2[C@@H]([NH2+]CCCN2CCCC2)C2CC2)cc1', 'O[C@@H]1c2cc(F)cc(Cl)c2C(=O)N1CCCCn1cccn1', 'ClCc1nonc1CN1C(=O)c2cc(Cl)ccc2S1(=O)=O', 'O=[N+]([O-])c1c(Cl)nsc1NC[C@H](O)c1ccccc1Cl', 'Cc1ccsc1S(=O)(=O)N[C@@H](C)CS(C)(=O)=O', 'O=C(c1ccsc1COCC1CC1)N1CC[NH+](Cc2ccccc2)CC1', 'ClC(c1ccc(F)cc1)[C@@H](O)c1cc(Br)cc(F)c1', 'Clc1ccccc1Oc1cc(N2Sc3ccccc3CC2)n[nH]1', 'O=C([C@H](c1c(Cl)cc([O-])cn1)N1CCCCC1)N1CCN(Cc2ccco2)CC1', 'O[C@@H](CSc1n[nH]c2c1CCCC2)c1ccccc1O', 'Oc1ccc([N+](=O)c2ccco2)cc1O', 'CNc1[nH+]ccc(N)c1S(=O)(=O)NNc1cccc2ccccc12', 'Clc1ccccc1C1(O[C@H]2CCS(=O)(=O)C2)CCCC1', 'O=[N+]([O-])c1c([O-])n[nH]c1NCc1cc2ccccc2o1', 'O[C@@H]([C@@H]1COc2ccccc21)c1cc(F)ccc1F', 'O=C(c1cc([N+](=O)[O-])ccc1F)N1CCC([NH2+]CCN2CCCCC2)CC1', 'Oc1ccccc1CN[C@H](CO)c1cc([N+](=O)c2cccnc2)ccc1F', 'Clc1nncc(O)c1', 'Oc1cc(OC2CC[NH+](Cc3ccccc3F)CC2)cc(SCC(=O)NC2CCC2)c1', 'Oc1cc(Cn2c[nH+]c(-c3ccccc3)n2)ccc1Cl', 'Clc1nnc(S[C@H]2CC(=O)N(c3ccc(F)cc3)C2)n1Cc1ccco1', 'C[C@@H](N[C@@H](c1ccc(F)cc1)c1n[nH]c(N)n1)c1ccncc1', 'O=[N+]([O-])c1ccc(/N=C\\c2nc(Nc3cccc(F)c3)c(=O)[nH]c2[O-])cc1', 'Oc1ccc(/C=N\\C[C@@H]2CSc3ccccc32)cc1', 'O=C([O-])c1nc([C@@H]2CCCN(c3ccccc3)C2)nc2c(C)cccc12', 'N=Cc1c(-c2ccccc2)nnnc1Sc1ncnc(N2CCC(C)CC2)c1CC', 'O=C(CNC(=O)C[NH+]1CCN(c2c(F)cccc2F)CC1)Nc1cccc(Cl)c1', 'Oc1cc(F)c2nc([C@H]3CCCN3C(=O)Cc3ccco3)oc2c1', 'O=C(c1cn[nH]c1)[C@@H]1C[C@@H]1c1ccc(Br)cc1', 'CNS(=O)(=O)C[C@H](C)ONC(=O)c1ccc2c(c1)CCO2', 'Oc1cc(-n2ncc(-c3ncnn3CCOc3ccccc3)n2)cc(S[C@H]2CCOCC2)c1', 'Oc1cc(F)c(Br)cn1', 'Clc1cc(F)cc([C@H]2CCCN2CCn2cnnc2)c1', 'C=C[C@@H]1N[C@@](CN2C(=O)C[C@@H](NCc3ccc[nH]3)N2)c2ccccc21', 'Clc1ccccc1N1NCC2(CCCCC2)O/C1=C/c1ccc(F)cc1', 'COC[C@H](C)OC(=O)CN1C(=O)c2c(C)cccc2N1c1cccc(Cl)c1', 'O=C(c1cc(Cl)c(Cl)c(Cl)c1)N1CCN(c2cccc[nH+]2)CC1', 'Clc1nncn1c1c(Cl)nc(Cl)cc1Cl', 'O=[N+]([O-])c1ccnc2c1CN(C(=O)CCCc1cccs1)C[C@@H]2O', 'Clc1ccc(-c2csc3[nH]nnc23)cn1', 'C=C(Br)Cn1c(SCC(=O)N[C@H](C)c2ccccc2)nn(-c2ccccc2)c1=S', 'Oc1cc(O[C@@H]2CCCc3ccccc32)c2ccccc2c1[O-]', 'Oc1ccc([N+](=O)[O-])c(I)n1', 'Oc1ccc([N+](=O)[O-])c(I)n1', 'Clc1ccc(Br)cc1O[C@H]1C=CCCC1', 'Clc1nnc([S-])n1-c1cccc(F)c1', 'OC1=CC(=O)CC1', 'O=C(c1ccc[nH]c1=O)N1N=C(c2cccs2)C[C@H]1c1cccnc1', 'Clc1ccc([C@@H](N[C@@H](C#N)Cc2nc3ccccc3[nH]2)C2CC2)cc1', 'O=C(CSC(O)(CO)OCc1ccccc1)NCc1cccs1', 'N#Cc1csc(NCc2cnc3ccccn23)c1', 'Oc1cc(-n2nc(Cl)cc2SCC(F)F)nc2ccccc12', 'Oc1ccc([C@H]2C=C(c3ccc(F)cc3)Nc3ncnn32)cc1', 'O=[N+]([O-])c1c([O-])[nH]c(=O)c2ccc(NC(=O)/C=C/N3CCCC3)cc12', 'Clc1cc(/C=N\\Oc2cccc(Br)c2)cc([O-])c1Cl', 'O=C(/C=C/Oc1ccc(Cl)cc1Cl)NC[C@@H](Cc1c[nH]c2ccccc12)[NH+]1CCCCC1', 'O=C(/N=c1/o/[n+](=N/C(=O)OCc2ccccc2)nc1-c1cccs1)c1ccc(Cl)cc1', 'Oc1ccc(O[C@H]2C=C[C@H](c3ccccc3)C2)cc1Cl', 'Oc1ccccc1O[C@@H]1CCC[C@@H](C(=O)[O-])C1', 'ClCc1n[nH]c2nccc(-c3ocnc3)c12', 'ClCc1csc(OC[C@H]2CCCO2)n1', 'O=[N+]([O-])c1c(OC[C@H]2CCCO2)ccnc1Sc1ncccn1', 'O=C([C@H](Sc1nccc(Cl)c1F)N1CCCC1)c1ccc2c(c1)OCCO2', 'N#C[C@H](c1ccccc1)[C@@H]1CCCN1Cc1ccc(F)c(F)c1', 'CC[S@](=O)[C@H]1CCCC[C@@H]1NC(=O)Nc1ccn(Cc2ccc(Cl)cc2)n1', 'O[C@@H](Sc1ccc(F)cc1Cl)Cn1ccnc1', 'O=[N+]([O-])c1cc(OC[C@@H]2CCCO2)cs1', 'Clc1ccc(C[NH2+]c2nc(C3CC3)n(C3CCCC3)n2)cc1', 'Clc1cc(Cl)c2scnc2n1', 'O=C(c1cc(F)ccc1F)N1CCC[C@@H]1CSc1ncnc2c1cnn2CC(F)(F)F', 'Oc1cccc(-n2cccc2C[NH+]2CCC(N3CCCC3)CC2)c1', 'Clc1cccc([C@@H](NCc2cc3nc[nH]c3c(Cl)c2Cl)C2CC2)c1', 'Oc1cccc(F)c1C[NH+]1CCC(OCc2ccccc2F)CC1', 'O=C(/C([O-])(c1ccc(F)cc1)C(F)(F)F)c1c(Cl)cccc1Cl', 'Clc1cc(Br)cc2nnnn12', 'ClC([O-])/C=C/c1ccc2nc(-c3cccc(F)c3)[nH]c2c1', 'ClCc1nnsc1-c1ccc(Cl)c(Cl)c1O', 'Clc1ccccc1O/C=C1/C(=N/CC2CCCCC2)N=N1', 'Clc1cccc(S(=O)(=O)Nc2ccc(C[NH+]3CCCC3)cc2)c1', 'Oc1ccccc1CCN1CC[NH+](Cc2cc(F)ccc2F)CC1', 'Clc1cccc2[nH]nc(NC[C@@](CO)[N+](=O)[O-])c12', 'COc1nn[nH]c1/C(C)CCc1cccc(F)c1', 'Clc1cccc(-c2n[nH]/c(=C/c3ncccc3Cl)o2)c1', 'Cc1cnc2ncc(C(=O)OCC#C/C=C/c3ccc(C#N)c(Br)c3)cc2c1', 'O=C(CS(=O)(=O)[C@@H]2CCS(=O)(=O)C2)N1Cc2ccccc2C1', 'COc1nc(C2(NC(=O)c3cc(C(C)C)no3)CCCC2)cs1', 'Clc1ccccc1O/N=C1\\SC(=C(Cl)Cl)N1c1cccc(F)c1', 'ClC(F)(C(F)F)Nc1ccccc1C(F)(F)F', 'O=C(Cc1ncccc1O[C@H]1CCSc2ccc(Cl)cc21)c1ccccc1', 'ClCc1nnc([C@@H]2COc3ccc(Cl)cc3C2)n1CC1CC1', 'NS(=O)(=O)Cc1noc([C@@H]2CCCN(c3ncccn3)C2)n1', 'N[C@@H](O)CCN1CCCCC1=O', 'Clc1ccc([C@@H]([O-])C[C@H](O)Cc2cc(Cl)ccc2F)cc1', 'N[C@@H](Sc1ccc(OCc2ccccc2F)cc1)c1ccn(C(F)F)n1', 'Clc1nncnc1N1CCCC[C@@H]1Cn1cncn1', 'Clc1nncnc1[N-]c1ccc(OCc2ccccn2)cc1', 'Oc1ccccc1[C@H](O)[C@@H]([C@H]2CCOC2)N1CCN(c2ccccc2F)C1', 'Clc1cc(Cl)cnc1N/N=C1\\N=C(c2ccco2)S1', 'O=[N+]([O-])c1ccc(COc2ccc(CC[NH+]3CCOCC3)cc2)cc1', 'C=C[C@@H](C)C(=O)NCCNC(=O)c1n[nH]c(C)c1C', 'Cc1noc2nc(CN3CCO[C@@H](OCc4ccc(F)cc4)C3)c(C#N)cc12', 'O[C@@H](S[C@@H](Cc1ccccc1)c1ccccc1)c1ccco1', 'Clc1cc(Cl)nnc1CSc1ncc[nH]1', 'COCCn1c(S[C@H](C(N)=O)C(=O)Nc2ccc(Cl)cc2)c(-c2ccccc2)c(C)n1', 'O=C(c1cc([S@](=O)Cc2ccccc2F)c[nH]1)Nc1ccc2nc(C3CC3)oc2c1', 'C[C@@H](c1nc(-c2cccnc2)n[nH]1)S(=O)(=O)c1ccc([N+](=O)[O-])cc1', 'CCn1c(C2CCCCC2)ncc2c(C)nn(CCC(=O)Nc3cccc(F)c3Cl)cc21', 'Oc1ccc([N+](=O)[O-])c(/N=N\\Nc2nnc(SCC(F)(F)F)s2)c1', 'N=CC1CCN(C(=O)Nc2c(Cl)cccc2Cl)C1', 'Nc1[nH]nc(N/N=c2/[nH]c3ccccc3n2Cc2cccs2)c1C(F)(F)F', 'Clc1ccccc1Oc1cnc2cc(-c3cccc(C4O[C@H](CO)C4)c3)[nH]c2n1', 'O[C@@H]1c2cc([N+](=O)[O-])ccc2Nc2cc(O)ccc21', 'Clc1nnn(-c2ncnc3sc4c(c23)CCC4)n1', 'O=C([C@H](c1c([N+](=O)[O-])cccc1[N+](=O)[O-])N1CCSCC1)N1CCCC[C@H]1C1CC1', 'Clc1cc(/C=C(/Cl)[C@H]2COc3ccccc32)cc([N+](=O)[O-])c1', 'Clc1cc(NC[C@@H]2CCS(=O)(=O)C2)ncn1', 'N=CCNC(=O)c1cncc2c1CCO2', 'O=C(COCC1CC[NH+](c2ccc(Cl)c(F)c2)CC1)/C=C/c1ccccc1', 'Clc1nnn(-c2cn(Cc3cccc(C(F)(F)F)c3)cn2)c1C#N', 'N[C@@H](CC[NH+]1CCN(c2ccccc2)CC1)c1ccc(Cl)cc1', 'CNS(=O)(=O)CN(C)C(=O)C[C@@H]1COc2ccccc21', 'ClCc1csc([C@H]2CCCO2)n1', 'O=[N+]([O-])c1ccncc1-n1cc([O-])c(NC2CC2)c1', 'O=C(c1cn2ccsc2n1)N1CCC[C@H](CSc2ccncc2)C1', 'Clc1cc(Br)c(CN2CCCC[C@H]2c2cnco2)cc1Cl', 'O=[N+]([O-])c1c(Br)cnc2nccn12', 'COc1nnnn1-c1ccc2nnnn2c1', 'Clc1cc(/C=N/O[C@H]2CC=CCC2)ccc1Cl', 'Clc1ccc(Br)c(/[NH+]=C\\c2ccccc2)c1', 'Clc1ccc(Br)c(/[NH+]=C/c2ccccc2)c1', 'Clc1cccc(Br)c1O[C@@H]1c2ccccc2C[C@H]1O', 'Clc1nnn(C[C@@H]2CCCC[C@@H]2Cc2ccc(F)cc2)n1', 'Oc1ccccc1CC[NH2+]Cc1ccc([N+](=O)[O-])cn1', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(O)c(=S)n1Cc1cccs1', 'NS(=O)(=O)/N=C(\\C=C1CCCC1)Nc1ccc2[nH]c(=O)[nH]c(=O)c2c1', 'O=C([O-])/C(=N/c1c(Cl)cccc1[N+](=O)[O-])N1CCCC1', 'O[C@@H](C[C@H]1CCCN(S(=O)(=O)c2ccc(C(F)(F)F)cc2)C1)c1cccnc1', 'O=C(/C=C1/NNc2ccccc2N1)N1CCC(OC[C@H]2CCCCO2)CC1', 'O[C@@H](Sc1nncn1Cc1ccccc1)Cc1nnc2ccccn21', 'Oc1cc(O[C@@H](C(=O)[O-])c2cc(=O)[nH]c3ccccc23)ccc1O', 'OC[C@@H](S)c1cc(Br)ccc1Cl', 'Clc1ccc(-c2n[nH]c3c2[C@H](c2ccccc2)NC3=O)c(O)c1Cl', 'Clc1ccccc1CS(=O)(=O)N1CCC([NH2+]C2CCN(C(=O)CCCc3ccccc3Cl)CC2)C1', 'O=C(c1c[nH]cn1)O[C@@H]1CCN(c2cccc(F)c2)C1', 'Clc1ccccc1OC1(Nc2c(F)ccc(Br)c2)c2ccccc2O1', 'Oc1ccc([N+](=O)[O-])c(C[NH+]2CCC(Oc3ccccc3)CC2)c1', 'COc1nn(-c2ccccc2)c(C(=O)N[C@H]2C[C@]2(c2ccc(Br)cc2)c2ccco2)c1C', 'O=C(CCCC[NH3+])C1CCCCC1', 'N#C[C@@H](Sc1cnccn1)Cc1cc(F)cc(F)c1', 'Oc1ccc(F)cc1C[C@@H]([NH3+])c1ccccc1F', 'N[C@@H](S/C=C/CO)c1ccc(Cl)cc1', 'Clc1ccc(CC(=O)[C@@H]2CSCN2C(F)(F)F)cc1', 'Clc1ccsc1N[C@H]1CCOc2c(F)cccc21', 'Clc1ccccc1[C@H](N)C(=O)[O-]', 'NS(=O)(=O)N(C)[C@H]1CCC[C@H]1c1cccc(Cl)c1', 'Clc1nnnn1/N=C/c1ccc([N+](=O)[O-])cc1O', 'Oc1cc(O[C@@H](C[NH+]2CCCC2)c2ccccc2)c(Cn2cccn2)c2ccccc12', 'O=[N+]([O-])c1c(Br)scc1C(F)(F)F', 'Clc1ccccc1CO[C@H](O)c1cscc1[N+](=O)[O-]', 'O=C(CC1([C@@H](O)c2ccccc2)CC1)N1CCC2(CC1)OCCO2', 'OC[C@@]1(COc2ccccc2C[NH2+]C(C)(C)C)CCCC1', 'Nc1cnn([C@@H]2CCC(=O)N(C3CCCCC3)C2)c(=O)n1', 'ClCc1cn[nH]c1NC[C@H]1CNc2ccccc2C1', 'CN[C@H](C(=O)n1ccc(=O)[nH]c1=O)c1ccc(C)c(C)c1', 'O=[N+]([O-])c1c(Br)c(O)c2c(c1C(N)=O)CC2', 'Oc1ccc([N+](=O)C=O)cc1Cl', 'Clc1ccc(-c2cc(S(=O)(=O)C[C@@H](O)c3ccncc3)no2)cc1', 'O=C(c1c[nH]c([N+](=O)[O-])c1)c1ccc(Br)s1', 'Oc1ccc(F)cc1C[NH2+]Cc1cc(Cl)ccc1O', 'CNS(=O)(=O)c1ccc(C(=O)NC[C@@H](c2ccc(Cl)c(Cl)c2)[NH+](C)C)cc1', 'O=C(/C(c1cccs1)Sc1ccccc1)c1ccno1', 'Clc1ccc(Br)c(Cl)c1C[NH2+]Cc1cccc([N+](=O)[O-])c1', 'Clc1cc(/C=N\\N2CCC[C@@H](C(=O)Nc3ccc([N+](=O)[O-])cc3)C2)ncn1', 'Clc1cc(Cl)c2sc(N[C@H]3CCS(=O)(=O)C3)nc2c1', 'Oc1cc(C(=O)N[C@@H]2COc3ccc(Cl)cc32)cs1', 'Clc1nncc(F)c1NC[C@H]1CCCO1', 'Clc1ccsc1SC[C@@H](O)c1cccc(Br)c1', 'O=[N+]([O-])c1c(F)cccc1NC1CC[NH+](Cc2ccc(Cl)cc2)CC1', 'O=C(c1cc(Br)ccc1O)N1CC[C@@](O)(C(F)(F)F)C1', 'Clc1cccc2cc(CNC[C@@H]3COc4ccccc4O3)nn12', 'ClCc1cncn2cnnc12', 'Clc1ccc(-c2nonc2)nn1', 'Clc1ccc(-c2nonc2)nn1', 'C[C@@H](c1ccsc1)N(C)C(=O)c1coc(C2CC2)n1', 'O=C(COc1cc(N(C(C)C)[C@H]2CCCC2=O)oc1C#N)N[C@@H](c1ccccc1)c1ccccc1', 'ClC[C@@H]1Oc2ccccc2C[C@@H]1O', 'ClCc1csc(S[C@@H](CO)c2ccc(F)c(F)c2)n1', 'OCc1nc([C@H]2CCO[C@H]2c2ccccc2)no1', 'ClC[C@@H](CO)c1cccs1', 'Clc1cc(Br)cc(Cl)c1OCC[NH3+]', 'O=[N+]([O-])c1c(Cl)nnc2sc(-c3ccco3)nc12', 'O=[N+]([O-])c1c(OC[C@H]2CCCO2)[nH]c2ccc(Cl)cc12', 'O=[N+]([O-])c1cc(SCCC(=O)N2CCC[C@@H](c3nc4ccccc4s3)C2)ncn1', 'COc1ncc(F)c(C[NH2+]Cc2ccc(SC)cc2)c1C#N', 'O=C(c1cn2ccsc2c1)N1CCC[C@H]1c1cccc(F)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1C(=O)N[C@H]1CCSc2ccc(F)cc21', 'Nc1[nH]nc(S[C@H](C(=O)[O-])c2ccccc2)c1-c1ccccc1', 'CN[C@H]1c2ccc(OC)c(C)c2C(=O)N(CC(C)C)C1', 'Clc1nnnc(NN2CCOCC2)n1', 'Oc1ccc([N+](=O)C=S)cc1', 'ClC(F)(F)COCO[C@H]1CCOc2ccccc21', 'Clc1cccc2cc([C@H](Br)[C@@H]3COc4ccccc4O3)ccc12', 'O=[N+]([O-])c1ccn([C@H]2CCCN2S(=O)(=O)c2ccccc2Br)c1', 'CCc1c(C)sc(NC(=O)c2c3ncn(CC(C)C)c3sc2C)c1C#N', 'O=C(c1c[nH]cn1)[C@H](Nc1ccsc1)c1ccccc1', 'Oc1ccccc1[C@@H](Cc1ccc(Br)s1)N1C[C@H]2CCCC[C@@H]2C1', 'Clc1ccc(Br)c(/C=C2\\N=C(c3ccccc3)N=N2)c1O', 'Clc1ccc(-c2cc[nH]n2)nc1SC[C@@H]1CN(C2CC2)CCO1', 'Oc1ccc(F)cc1O/N=C/c1ccc(S(=O)(=O)[N-]c2cccc(Cl)c2)cc1', 'COC[C@H](C)NC(=O)[C@@H]1C[C@H]1c1cc(Cl)ccc1Cl', 'O=[N+]([O-])c1ccc[nH+]c1N1CCC(NC2CCOCC2)CC1', 'N#C/C([O-])=C(\\CC(=O)c1ccc(F)cc1)c1cc(C#CCCO)ccc1O', 'COC[C@H]([NH3+])Cc1ccc(OC(F)F)cc1', 'Clc1ccc(-c2cc[nH]n2)nc1NC[C@H](O)COc1cccnc1', 'O=C(CSCC(F)(F)F)N[C@@H](C(=O)c1cccc(Cl)c1)N1CCCC1', 'N=C1\\C[NH+](CCCCN2CCCCC2)[C@H](C)C(=O)N1c1ccccc1', 'Clc1cc(Cl)cnc1[N-]c1ccc2ccccc2c1', 'Clc1ccc([C@@H]([NH2+][C@@H]2CCOC2)c2ccc(Cl)cc2)cc1', 'Oc1cc(F)ccc1C[NH2+]Cc1ccc(F)cc1F', 'Oc1ccc(/C=C2\\CCS(=O)(=O)C2)c(Br)c1', 'O=C([O-])CC[C@@H]1COc2ccccc21', 'O=C(Cc1cnoc1)N1CCC(CC[NH+]2CCCCC2)CC1', 'Oc1ccc(O[C@@H]2C[C@@H]3CCCC[C@H]3C2)cc1OCC(F)(F)F', 'O=[N+]([O-])c1cc(SC(CC2CC2)C(=O)[O-])ccc1Br', 'O=C([C@H](Nc1ncccn1)C1CC1)NC1(CC(F)(F)F)CCCC1', 'Clc1ccc(-c2cc[nH]n2)c(C[NH2+]CCc2ccccc2)c1', 'Oc1ccc(/N=c2/[nH]c(-c3cccc(Cl)c3)n[nH]2)c(Br)c1', 'O=[N+]([O-])c1ccnc2ccc(S(=O)(=O)C[C@H](O)c3cccs3)cc12', 'Clc1cc(Cl)nn1/N=N\\N1CCN(c2ncc(-c3ccccc3)s2)CC1', 'O=C(COc1ccc(NC(=O)C[C@H]2CSc3ccccc32)[nH]1)Nc1ccccc1', 'Clc1nncc(Sc2nnc3ccc(F)cc3c2Cl)n1', 'NS(=O)(=O)C[C@H]1Oc2ccccc2C(=O)N1c1ccc(F)cc1F', 'O=[N+]([O-])c1ccn(C[C@H](O)c2cccs2)c1', 'Cc1sc2nc(/C=C/c3cc(N)c(C#N)cn3)[nH]c(=O)c2c1C', 'O=[N+]([O-])c1ccncc1O[C@@H]1CCS(=O)(=O)C1', 'Clc1cc(Br)cnc1[C@H]1CCCN1S(=O)(=O)c1cccs1', 'O=C([O-])c1nnnn1-c1cccc(OCC(F)F)n1', 'C=C[C@@H](S(=O)(=O)c1ccc(C)cc1)c1nc(C)cc(C)c1[N+](=O)[O-]', 'Clc1ccc(C2C[C@](NC3CC3)/C(=C/Cl)C2)cc1', 'O=C([O-])/C=C(\\Nc1ccc(Br)cc1)N[C@@H]1CCOC1', 'Oc1cc(Cn2ccnn2)c2cn[nH]c2c1O', 'NS(=O)(=O)Cc1noc(CC[S@@](=O)Cc2ccccc2)n1', 'Clc1cc(C[NH2+]C2CC2)ccc1N1CCOCC1', 'Clc1ccc([C@@H]([NH2+]Cc2n[nH]c3ccccc23)C2CCOCC2)cc1', 'O=[N+]([O-])c1ccncc1N[C@@H]1c2ccccc2CC[C@@H]1O', 'N[C@@H](O)c1csc(N2CCCC2)n1', 'ClCc1csc(OC[C@@H]2C[C@H]2c2ccccc2)n1', 'C[C@@H](Nc1cccc(C#CN2CCOCC2)c1)c1nnc(-c2ccccc2)n1C', 'Clc1cccc(S[C@H](C(=O)[O-])C2CCOCC2)c1', 'Cc1snnc1C(=O)N1CCc2cc(O)ccc2[C@H]1c1cccc(O)c1', 'O=C(CC1([NH3+])CCC1)N1CCCCC1', 'C[C@@H](N[C@@H]1CC(C)(C)COC1)C(=O)N[C@@H]1CCN(c2c(F)cccc2F)C1', 'Oc1cc(CCNC2(CS)CCOCC2)ccn1', 'ClCc1cn[nH]c1[C@H]1CCCN1c1ccc2c(c1)OCO2', 'O=[N+]([O-])c1ccncc1N1C[C@H](O)CC[C@@H]1c1ccc(Cl)cc1', 'Clc1nnn(C[C@@H]2CCc3ccccc32)c1Cl', 'Clc1ccccc1-c1cnc2n1CCC[C@@H]2N[C@@H]1CCS(=O)(=O)CC1', 'Oc1ccc(O[C@H](Cc2cscn2)C(=O)c2cccc3ccccc23)cn1', 'O=C(CS(=O)(=O)[N-]C1CCCC1)Nc1nc2ccccc2s1', 'O=C(/C=C/c1ccco1)N1CCC[C@H](C2(O)COC2)C1', 'N[C@@H](S/C=C/c1ccc(Cl)cc1)c1nccn1C', 'Clc1cccc([S@@](=O)Cc2c[nH]nc2-c2ccccc2)c1', 'OCS1(=O)CCN(C(=O)COc2ccccc2)CC1', 'Clc1nnnn1/N=C/c1ccc([N+]2=C(CCCc3ccccc3)C(=O)NC2=S)cc1', 'O[C@@H](S[C@@H](c1ccccc1)c1ccncc1)C1CC1', 'O=C([C@H](Cl)Br)c1cccs1', 'Clc1ccccc1[C@H](Cl)[C@](Cc1ccccc1)[NH+]1CCCC1', 'O=C(/C(c1ccnc(Cl)c1)N[C@H]1CCN(c2ccccc2)C1=O)c1cccnc1', 'O=[N+]([O-])c1c(OCc2ncon2)oc2ccc(Cl)cc12', 'ClC[C@H]1OC(c2ccccc2)=C1Cl', 'Clc1ccccc1NS(=O)(=O)NC[C@@]1(O)CCCO1', 'Oc1ccccc1[C@@H]([NH2+]Cc1ccco1)c1ccccn1', 'N=C1CN(C(=O)[C@H]2CCCN(C(=O)C3COC3)C2)c2ccccc21', 'O=C(c1ccc(F)c(F)c1)N1CCSC[C@@H]1C(=O)[O-]', 'O=[N+]([O-])c1cccc2c1NC(=O)C[C@H]2c1ccccc1NCC1(Cl)CCCC1', 'O=C(/C=C1/N[C@H](c2ccccc2)SC1=S)c1ccc(F)cc1', 'Clc1ccc(-c2cc(F)c(/C=C/C(=O)[O-])o2)cc1', 'O[C@@H]([C@@H]1Cc2ccccc2O1)C1(c2ccccc2)CCOCC1', 'Clc1ccccc1[C@H]1C[C@@H](c2cccc(F)c2)CS1', 'O[C@@H]1c2ccc(Br)cc2C[C@H]1CC(=O)Nc1ccc(OCC=O)cc1', 'Clc1cccc(Cn2ncc3c2NC23CC2)c1', 'Clc1ccc(-c2cc[nH]n2)o1\n\n\n\n\n', 'Clc1ccc(-c2cc[nH]n2)o1\n\n\n\n', 'Clc1ccc(-c2cc[nH]n2)o1\n\n\n', 'Clc1ccc(-c2cc[nH]n2)o1\n\n', 'Clc1ccc(-c2cc[nH]n2)o1\n', 'Clc1ccc(-c2cc[nH]n2)o1', 'Clc1ccc(-c2cc[nH]n2)o1', 'Clc1ccc(-c2cc[nH]n2)o1', 'O=[N+]([O-])c1c(F)cc(C(=O)[O-])cc1F', 'Oc1ccc(F)cc1[C@H]1Oc2ccccc2C(=O)N1C[C@@H](O)C1CC1', 'OC[C@H](Cc1ccccc1)Oc1ccc(N2CCOCC2)[nH+]c1', 'Clc1ccc(-c2n[nH]c3[nH]c[n+]([O-])c23)cc1', 'O=[N+]([O-])c1cc(Br)c([O-])cc1NCC(F)(F)F', 'O=[N+]([O-])c1c(-c2cc(F)ccc2O)n[nH]c1[O-]', 'Clc1nncc(F)c1F', 'Oc1ccc(O[C@@H](Cc2nc3cc(Cl)ccn3c2C(F)(F)F)C2CCCC2)cc1', 'Nc1[nH]nc(N2CCC[C@@H](c3nc4ccccc4[nH]3)C2)c1N', 'N#Cc1nc(-c2ccsc2)nc2c1CC[C@@H](c1ccccc1)C2', 'Clc1cc(Cl)ncc1Sc1ncccc1C(=O)[O-]', 'O=C([C@H]1CC(=O)N(C2CCCC2)C1)N1CCCC[C@H]1CO', 'Oc1cc(C(F)(F)F)cs1', 'O=C(c1cnc([C@@H]2CCCC[C@@H]2C(=O)N2CCCC2)s1)N1CCOCC1', 'NS(=O)(=O)C[C@H](c1cccs1)n1nnc2cc(Cl)ccc21', 'Nc1nncn1[C@@H]1CC(=O)N(c2cccc(Cl)c2)C1', 'Clc1ccccc1[C@H]1NC(=O)NC(=S)N1', 'O=C(Cc1n[nH]c2ncccc12)N1CCC[C@H](c2nc(C(F)(F)F)cs2)C1', 'Clc1ccccc1-c1nc(C[S@@](=O)C/C=C/c2ccc(Br)cc2)no1', 'Nc1[nH]nc2c1-c1nc(N)ccc1N2', 'OC[C@H](SC1=NCCS1)c1ccccc1', 'Clc1ccccc1C[NH+]1CCN(c2c(Cl)cccc2[N+](=O)[O-])CC1', 'Nc1nccs1', 'Clc1cccc(NC[C@]2(O)CCNC2=O)c1', 'CCC[NH2+]Cc1cc(S(C)(=O)=O)ccc1[N+](=O)[O-]', 'NS(=O)(=O)N(Cc1ccc[nH]1)C[C@@H]1CCCCO1', 'Clc1cc(NSc2ccc(Cc3nc[nH]n3)o2)ccc1OCc1ccc(Cl)cc1', 'Clc1ccsc1-c1nnc(CSc2ncccn2)[nH]1', 'Cc1nonc1C(=O)N(CC1CCOCC1)[C@H]1CCS(=O)(=O)C1', 'Oc1cc(COC[C@@H]2CC[NH+](Cc3ccccc3)C2)ccc1OCc1ccccc1', 'Clc1ccccc1OCC[NH+]1CCC[C@H]1c1ccc(F)cc1', 'O=[N+]([O-])c1cc(C(=O)Nc2ccc(Cl)cc2)ccc1N/N=C/[C@@H]1C=CC=C1Cl', 'O=C(c1ccccc1F)N1CCC[C@@H](n2nccc2CS(=O)(=O)C(F)(F)F)C1', 'Oc1cccc(N2C(=O)[C@@H](Cc3ccccc3)N2c2ccc(C(=O)[O-])cc2)c1', 'C[C@@H]([C@@H](O)c1ccccc1)N1CCc2ncc(C(F)(F)F)cc2C1', 'Clc1ccccc1NCC[NH+]1CCC(COS(=O)(=O)c2ccccc2)CC1', 'Clc1cc(Br)cn1Nc1ncccc1F', 'Oc1ccccc1[C@@H]1C(c2cccs2)=CN=C1c1ccc([N+](=O)[O-])cc1', 'Oc1cc(F)nc(Cl)n1', 'Clc1ccccc1C1=CCN=CN1', 'Clc1ccccc1OC1(Oc2cnc(-c3cscn3)s2)CCOCC1', 'Clc1ccccc1-c1cscc1C[NH2+]c1ccccn1', 'Clc1ccc(Br)cc1/C=N\\O[C@@H]1CCCO[C@@H]1c1ccccc1', 'O[C@@H](COCc1cc(-n2cccc2)c[nH]1)c1ccc(F)cc1', 'C[C@@H]([C@@H](O)CCO)N(C)C(=O)c1ccc(Br)cc1F', 'O=C(/C=C\\COC1CC[NH+](Cc2cccs2)CC1)Nc1cccc2c1CCCC2', 'Cc1noc2nc(S[C@@H](C)C(=O)c3ccccc3)sc12', 'O=[N+]([O-])c1c([N+](=O)[O-])cnnc1N', 'O=C(/N=C\\c1ccco1)c1ccccc1[N-]Sc1ccccc1', 'NS(=O)(=O)N1Cc2ccccc2S(=O)(=O)[C@@H]1Cc1ccccc1-n1cccc1', 'Clc1ccccc1NC(=O)[C@@H]1CC(c2ccccc2)N=[N+]1c1ccccc1', 'Cc1cc(Cl)ccc1NC(=O)CSC1=N[C@@H]2CS(=O)(=O)C[C@@H]2N1c1ccccc1', 'Clc1cc(Br)cnc1OC[C@H]1Cc2ccccc2S1', 'O=[N+]([O-])c1c([O-])nnnc1Sc1ccccc1', 'N#CCNC(=O)c1cc(C(=O)N2C[C@H]3CCCC[C@@H]3C2)nn1-c1ccccc1', 'COc1nn[nH]c1CC1CC[NH+]([C@@H](C(=O)N2CCCCC2)c2ccccc2)CC1', 'COc1nnnc(Sc2nnc(C)c(C)n2)c1C', 'CCC[C@]1(C)CCN(c2ccc([N+](=O)[O-])c([N+](=O)[O-])n2)C1', 'ClC[C@@H]1C[C@@H](O)c2ccccc21', 'Nc1c(/N=C/c2ccc3c(c2)OCO3)ncnc1N[C@H]1CCS(=O)(=O)C1', 'O=[N+]([O-])c1c(F)cc(NN)c(O)c1O', 'NS(=O)(=O)/C(=C\\NCc1ccccc1)n1cc(Br)c([N+](=O)[O-])c1', 'Clc1ccc(-c2cc(Br)oc2C2CC2)o1', 'Oc1cccc(-n2ccc3ncnn32)c1', 'Clc1cc(Br)c2nc(O)ncc2c1O', 'Clc1cc(Br)c2nc(O)ncc2c1O', 'O=C(c1c[nH]cn1)O/N=C/C(Cl)=C/c1ccc([N+](=O)[O-])cc1', 'Clc1nncn1c1c(C(F)(F)F)c(Cl)cn1C1CCC(O)CC1', 'Oc1ccccc1/N=N\\n1cnnc1', 'Oc1cccc2cc([C@@H](CNS(=O)(=O)c3ccc4c(c3)OCO4)C(=O)[O-])c(=O)n(Cc3ccccc3)c12', 'Clc1ccc(C2=CC[NH+](Cc3ccccc3)CC2)cc1', 'C[C@@H](OC[C@@H]1CCCO1)C(=O)OCC(=O)N1CCN(C)CC1', 'Clc1ccccc1-c1csc(C[NH2+]CCO)n1', 'Clc1nnnc(NC[C@@H]2CCCO2)n1', 'O=C(c1c[nH]nc1N[C@H]1CCSc2ccccc21)N1CC=C(c2ccccc2)CC1', 'Oc1ccc([N+](=O)C=[N+]2NC(=O)N(c3ccccc3)c3ccccc32)cc1', 'O=C([O-])[C@H](c1ccccc1)n1cnnn1', 'O[C@@H](Sc1cccs1)C(=O)NNS(=O)(=O)c1ccccc1Cl', 'Clc1ccc(CCC[NH2+]Cc2ccc[nH]2)cc1Cl', 'Oc1ccccc1/C=N/N(Cc1c(F)cccc1Cl)[C@@H]1CCS(=O)(=O)C1', 'Clc1ccc(Br)cc1/N=C1/NNc2ccc([N+](=O)[O-])cc21', 'Clc1ccc(C[NH+](C[C@@H]2Cc3ccccc3O2)c2nccn2Cc2ccccc2)cc1', 'Clc1ccc(Br)cc1S[C@H]1c2ccccc2C1(Cl)Cl', 'Clc1nnnc(Cl)c1-n1cccn1', 'Clc1nnnc(NN=CSc2ccc(Cl)cc2)n1', 'Clc1ccc(-c2csc3s/cnc23)nn1', 'Oc1ccc([N+](=O)[O-])cc1Sc1ccccc1C(=O)Nc1nc(Cn2ccc(C(F)(F)F)c2)n[nH]1', 'O=C(c1ccsc1C#N)[C@@H]1Cc2ccccc2C(=O)N1c1ccc(F)cc1', 'Clc1ccc(-c2nn([C@@H](O)[C@@H](CC(=O)NC3CC3)C(C#N)=C3CCCCC3)n2Cc2ccccc2)cc1', 'Clc1ccc(Br)c(Cl)c1[C@@H](CO)NC1(N2CCOCC2)CCCC1', 'Clc1ccc(Br)cc1[C@@H]([NH3+])COOCc1cccc(F)c1', 'Clc1ccc(Br)c(F)c1C[NH2+][C@@H]1CC[C@H](CCc2ccccc2)C1', 'O=[N+]([O-])c1cc(SC[C@@H]2Cc3ccccc3O2)c[nH]c1=O', 'Clc1nnnn1-c1c(C(F)(F)F)cc(C(=O)[O-])c2ccccc12', 'Clc1ccc(Br)cc1NC1=CSCCS1', 'Clc1ccsc1Sc1ccc(Cl)c(C(=O)[O-])c1', 'O=C(c1cc([N+](=O)[O-])[nH]n1)N(Cc1ccc(Cl)cn1)[C@H]1CCS(=O)(=O)C1', 'ClCc1csc([N-]Cc2nc(-c3ccccc3)no2)n1', 'Cc1snc(C)c1S(=O)(=O)N[C@@H](CO)c1cccc(Cl)c1', 'COc1nn(-c2c(F)cccc2Cl)c2c1[C@@H](c1ccc(OC)cc1C)C2', 'Clc1cc(NC[C@@](O)(c2cccc(F)c2)n2nc(C(F)(F)F)cc2)ccc1Cl', 'Oc1cc(F)nc([S-])c1SCc1ccccc1Cl', 'Clc1nnnc(OCCC[NH2+]Cc2ccccc2)c1[N+](=O)[O-]', 'Oc1cc(-n2nc3c(=S)[nH]c(-n4cncn4)c3c2=O)ccc1Cc1ccccc1F', 'O[C@@H](c1cc(Cl)ccc1O)c1c[nH]nc1C1CCCCC1', 'Cc1nonc1CC(=O)N1CCCC[C@@H]1C(=O)[O-]', 'N#C[C@@H](Nc1ncccc1O)c1cccc(-n2cccn2)c1', 'Oc1ccccc1-c1csc2nc(SCC(=O)OC[C@@H]3CCCO3)[nH]c12', 'Clc1cc(/N=C/C=C/c2ccccc2)nc([O-])n1', 'Clc1cccc(F)c1C[C@@H](C[NH3+])c1c(F)cccc1F', 'O=C(C1=C([O-])C(=O)N(Cc2c[nH]c3ccccc23)C1=O)N1[C@@H]2CC[C@@H]1Cc1ccccc1-2', 'O=C(/C=C1CC=C(c2ccc(F)c(F)c2)C1)[C@H]1COc2ccccc2C1', 'O=C(/N=C/Nc1ccc(NSc2ncccc2F)cc1OC(F)F)Nc1ccc(C(N)=O)cc1', 'Oc1ccc([N+](=O)[O-])c(CSc2nnc[nH]2)c1Cl', 'N=Cc1nonc1S(=O)(=O)Cc1cccc([N+](=O)[O-])c1', 'Nc1nnc(C2=NN2c2cccc(C(F)(F)F)c2)n1/N=C/[C@H]1CCCC[C@@H]1C(=O)Nc1cccc2[nH]ccc12', 'Clc1ccc(-c2nonc2)nc1C1CC1', 'Clc1ccccc1O/N=[N+]/C=CSc1ccc2c(c1)OCO2', 'Nc1[nH]nc2c1SC/C2=C(\\N)Nc1ccccc1', 'Clc1cc(/N=C\\C2=CCNC2)c2ccccc2c1Cl', 'Clc1ccc([C@@H]([C@@H](Cl)c2ncc[nH]2)N2CCOCC2)cc1', 'O=[N+]([O-])c1c(Br)scc1[C@H](O)CS(=O)(=O)c1ccccc1', 'Clc1ccc(CCC2C=[NH+][C@]3(CCO/C(=C/C(=O)c4ccc5c(c4)OCO5)C3)[C@H]2C)cc1', 'N#CC1=C([O-])CC2=C(C(=O)N(Cc3cccc4ccccc34)C2)[C@@H]1c1ccccc1Cl', 'CCN[C@](CCC(F)(F)F)C(=O)OC', 'OCc1nnnn1[C@H]1CCCN(c2cc(F)cc(F)c2F)C1=O', 'NS(=O)(=O)/C(=C\\c1ccco1)CN1C(=O)CN(CCO)C1=O', 'Clc1ccccc1N1NC2(CCCCC2)N1', 'Clc1cccc([S@@](=O)Cc2ccc(-c3ccco3)s2)c1', 'N#C[C@@H]1CN(Cc2ccccc2Cn2cccn2)CCS1', 'COc1nn[nH]c1C(=O)C[C@H]1CCCN(c2ncccc2Br)C1', 'O=C(/C(c1cccs1)c1ccccc1Cl)[C@H]1C[C@H]1c1cccc(Cl)c1', 'O=C(/C([O-])=N/S(=O)(=O)c1ccccc1Br)c1[nH]ccc1C', 'Clc1ccc(-c2nonc2)nc1C(F)(F)F', 'Clc1cc(Cl)c(N2CCOC[C@H]2C2(CO)CC2)cc1Cl', 'Clc1ccccc1-c1nc(C[NH+](Cc2ccco2)Cc2ccco2)co1', 'Clc1nncnc1N', 'Nc1[nH]ncc1N(C(=O)c1ccc(Cl)cc1N(C)Oc1ccccc1Cl)S(C)(=O)=O', 'Oc1cccc2nc([C@H](c3ccccc3)N3CC[NH+](C[C@H]4CCCO4)CC3)ncc12', 'Clc1ccc(C2=C[C@H](S(=O)(=O)Cc3ccc(Br)cc3F)N2[N+](=O)[O-])cc1', 'C[C@@H](c1ncoc1C1CC1)c1nnc(NC(=O)c2cccc(Cl)c2)s1', 'O=[N+]([O-])c1c(F)cc(Nc2ncccn2)o1', 'O=[N+]([O-])c1c(F)cc(N[C@@H]2CCN(S(=O)(=O)OCCCn3cccc3)C2)cc1', 'Oc1ccc(F)cc1-c1ccccc1[C@@H]1CCCN(C(=O)c2cnnc3c2s3)C1', 'Oc1ccc(/N=c2/[nH]c3cc(Br)c([N+](=O)[O-])cc3s2)cc1O', 'ClC[C@H]1CNc2ccc(Cl)cc21', 'Oc1cccc(Br)c1C[C@H]([NH2+]Cc1ccc(Cl)cc1)c1cccc(F)c1', 'Oc1ccc(/C=C2/C(=O)N(c3ccccc3)SC2=S)cn1', 'COc1ccc(C[C@@]2(C)NC(=O)N(CC(=O)N(C)S(=O)(=O)Cc3ccco3)C2=O)cc1', 'Clc1ccc(-c2nnnn2CC[NH+]2CCCC2)cc1', 'Oc1ccc(F)c([C@@H]2CSCCN2C(=O)C2=CCCCC2)c1', 'Clc1ccccc1[C@H]([NH3+])c1cnn2c1CN(Cc1ccccc1Cl)C2', 'ClC[C@@H]1OCC[C@@H]1c1ccc(F)cc1', 'ClC(/N/N=C\\C=C/c1ccccc1O)C(=O)Nc1ccccc1C(F)(F)F', 'O[C@@H]1[C@@H](O)[C@@H](O)[C@@H]1Cc1cc(Cl)ccc1Cl', 'Clc1ccccc1N1N=C(C(F)F)[C@@H](c2cccc(F)c2)C(=O)N1c1ccccc1Cl', 'Clc1cc(/N=C/c2n[nH]n3cccc23)ccc1O', 'O=C(c1ccc(OS(=O)(=O)c2ccco2)cc1)[S@@](=O)c1ccc(Cl)cc1', 'O=[N+]([O-])c1c(Cl)nc([C@@H]2CCCC[C@@H]2O)nc1-c1ccccc1', 'Clc1ccc([C@@H]([NH2+][C@@H](CO)c2ccccc2)C2CC2)cc1', 'Clc1ccccc1-c1cscc1C[NH2+]Cc1nnc2n1CCC2', 'Clc1ccccc1C1=NN=C[C@@H]1C(=O)NC1(c2ccccc2)CC1', 'O[C@@H]1CSc2ccccc2O1', 'ClCc1nc(I)cn1C1CC1', 'O=C(CN1CNc2[nH]c[nH+]c2[C@@H]1c1ccccc1)NC[C@H](O)c1ccccc1', 'Oc1cc(O[C@@H](CC(F)F)c2c[nH]nc2-c2ccccc2)ccc1O', 'N=Cc1cncc(CN2CCCC[C@@H]2c2cccc([N+](=O)[O-])c2)n1', 'Clc1ccccc1Cc1ccc(O[C@@H]2CC[NH2+]C2)c(Br)c1', 'O=[N+]([O-])c1cc(Br)c2[nH]cnc2n1', 'O=C(c1ccsc1)N(C[C@@H](O)c1ccccc1)C[C@@H]1CN2CCC[C@@H]2CO1', 'O[C@@H](Sc1n[nH]c2c1CCCC2)c1cccc(OC2CC2)c1', 'Clc1cc(/C=N/N(c2ccccc2Cl)C2CC2)c[nH]1', 'O=[N+]([O-])c1ccc(O[C@H]2c3ccccc3C=C[C@@H]2O)cc1', 'Oc1ccc([N+](=O)C=NN2C(=O)N[C@](c3ccccc3)=N2)cc1', 'O=C(CC(CC1(Cl)CC1)NCCCS(C)(=O)=O)C(=O)Nc1ccccc1', 'Clc1cccc2cc([C@H](C(=O)NCc3cscn3)N3CCOCC3)sc12', 'Clc1ccsc1CN1CC[C@H]2[C@H](C2)C1', 'Clc1ccc(-c2ccccc2)cc1C[NH2+]C[C@H]1CCS(=O)(=O)C1', 'O=[N+]([O-])c1c(Br)sc2ncn(Cc3cccnc3C(F)F)c12', 'Oc1cc(Br)cs1\n\n\n\n\n\n\n', 'Oc1cc(Br)cs1\n\n\n\n\n\n', 'Oc1cc(Br)cs1\n\n\n\n\n', 'Oc1cc(Br)cs1\n\n\n\n', 'Oc1cc(Br)cs1\n\n\n', 'Oc1cc(Br)cs1\n\n', 'Oc1cc(Br)cs1\n', 'Oc1cc(Br)cs1', 'Oc1cc(Br)cs1', 'Oc1cc(Br)cs1', 'NS(=O)(=O)CC[C@H](c1ccccc1)c1n[nH]cc1C(C)=O', 'O=C([C@H](Sc1noc2ccccc12)C1CCCC1)NC[C@H](O)c1ccccc1', 'O=[N+]([O-])c1ccc2c(c1)[C@@H]1C=CCC[C@H]1[C@H](c1ccc(F)cc1)N2C1CCCCC1', 'Nc1c(CN(C)C(=O)[C@H]2CCO[C@H]2C)cnn1-c1ccccc1', 'NS(=O)(=O)N[C@@H]1CCCN(CC(F)(F)F)C1', 'Clc1ccc(Br)c(Nc2nnc3c(O)nc(SCC(F)(F)F)n3n2)c1', 'ClCc1cn[nH]c1[C@@H]1CCCN1c1ccc(F)cc1C(F)(F)F', 'Clc1cccc([C@@H]2[C@@H](C(=O)c3cc(Cl)cc(Cl)c3)NC2=O)c1', 'N#C[C@@H]1OC(=O)N(CCCN2CCc3ccccc32)C1=O', 'Nc1nonc1Cc1ccc[nH]1', 'O[C@@H](S[C@@H]1CCOC1)c1ccccc1', 'O=[N+]([O-])c1ccc([C@H](CCCO)N[C@H]2CC(c3ccccc3)=NO2)c2ccccc12', 'O[C@@H]([C@@H](CCc1ccsc1)NCCCc1ccc2c(c1)OCCO2)C1CC1', 'Clc1cc(Br)c2nc(OC[C@H]3CCCO3)ncc2c1O', 'Nc1nncn2cnnc12', 'O=C(c1cn(C[C@H]2CCCO2)cc1)N1CCC[C@@H]1CF', 'N#C/C(=C/c1cccs1)C(=O)Cn1cnc([N+](=O)[O-])c1', 'O=C(CNc1nn(Cc2ccncc2)nc1Cl)Nc1cccc2c1C[C@@H](O)C2', 'Cc1sc(=O)n(CCC(=O)N[C@H]2CCSC2)c1C', 'Clc1ccsc1NSc1ccc(Br)cn1', 'O=C(CS(=O)(=O)C[C@H]1Sc2ccccc2NC1=O)NC[C@@H](O)c1ccccc1', 'Cc1ccsc1-c1nnc(SCC(=O)[O-])s1', 'O=C(C1CCN(C2CCCC2)CC1)N1CCC2(CCSCC2)CC1', 'Clc1ccsc1NSc1ncccn1', 'Clc1ccsc1[C@@H]1CC(C(=O)N2CCN(CCO)CC2)=NO1', 'O=[N+]([O-])c1ccn(C[C@@H](Cl)Oc2cccc(F)c2)c1', 'Cc1c(-c2cccs2)ncnc1N1CCC([C@H]2CCN(C)C2=O)CC1', 'N#Cc1c(F)cc(CNC(=O)NC[C@H]2CC=CCC2)cc1C(F)(F)F', 'COc1ncnc(N[C@@H]2CCO[C@H]2c2ccccc2F)c1[N+](=O)[O-]', 'ClCc1csc2nc3c(cc12)CNC3', 'ClC(F)(Cc1ccco1)C(=O)c1ccc(Br)s1', 'COc1ncc(NC(=O)N2C[C@H](C)O[C@H](C)C2)c(C)c1OC', 'Clc1ccc2ccccc2c1Oc1ccccc1C[NH2+][C@@H]1CCS(=O)(=O)C1', 'ClC[C@H]1C[C@H]1c1cccc(F)c1Cl', 'Oc1ccc([N+](=O)[O-])c(S[C@@H](C(=O)[O-])c2ccccc2Cl)c1', 'O=C([O-])[C@@H]1CN([C@@H]2CCCc3c(F)cccc32)C1', 'CNC(=O)c1ccc([C@H](C)[NH2+]C2CCN(c3ccccc3F)CC2)cc1', 'ClCc1nnnn1[C@@H]1CCC[C@@H](C2CC2)C1', 'Clc1cc(NC2CC[NH+](Cc3ccc4c(c3)OCCO4)CC2)ccc1F', 'Cc1snnc1CN(C)C(=O)c1ccc([S@](C)=O)cc1', 'O=C(COCC1CCN(S(=O)(=O)c2ccccc2)CC1)N1N=C[C@@H](OC2CC2)c2ccccc21', 'Clc1ccc([C@@H](O)[C@H]([NH2+]Cc2ccccc2)C2CCC2)cc1', 'ClC[C@@H]1CNc2ccc(F)cc21', 'Nc1nncn1/N=N/N=C/c1ccc([N+](=O)[O-])cc1', 'COc1ncc2nc(NC(=O)Nc3cc(C(C)(C)C)cs3)nn2c1', 'O=C(/N=C/N=C\\c1cccc(O)c1)[C@H]1CCCN(S(=O)(=O)Cc2ccccc2)C1', 'Clc1ccccc1NC1=NN([C@@H]2CCS(=O)(=O)C2)C1', 'O=C(c1cc(F)ccc1F)N1CCC[C@@H]1[C@@H]1CCCC[C@@H]1O', 'O=C(CS(=O)(=O)[N-]Cc1cccc(Cl)c1)NCCC(F)F', 'Oc1ccccc1/N=C(\\N/C1=N/S=C1N1CCC[C@@H]1c1ccc(Cl)cc1)c1ccccc1', 'O=C([C@H](SCc1cccs1)N1C(=O)CSC1=O)Nc1ccc(Br)cc1', 'Clc1ccc(COCO[C@@H]2CCO[C@]2(F)c2ccc(F)cc2)cc1', 'Cc1cc(NC(=O)C(=O)N[C@@H](C)c2ccsc2)n(C)c1', 'Clc1cc(/N=C\\CCn2ccc(Br)c2)ccn1', 'NS(=O)(=O)Nc1cc([N-]S(=O)(=O)c2cc(F)ccc2F)ccc1F', 'O=C([C@H](Cl)C(F)(F)F)N1CCOCC1', 'Clc1cc(/C=C/[N+](=O)[O-])cc(C(=O)[O-])c1', 'O=C([O-])c1cnc(=S)n(-c2nccc(Oc3cccnc3)n2)c1', 'NS(=O)(=O)C[C@@H](F)c1cccnc1', 'Clc1ccsc1SCC1(Cn2ccnc2)CCOCC1', 'Oc1ccc([N+](=O)c2oncc2C2CC2)cc1F', 'Clc1cc(F)c(NCCCC[NH2+]C2CC2)cc1', 'Clc1cccc2ccc(NCCC[NH+]3CCCC3)nc12', 'Clc1cccc2c1[C@@H](Cn1ccnc1-c1ccc[nH]c1=O)C2', 'Clc1nnnn1CCSc1nc(CC2CC2)n[nH]1', 'Clc1ccccc1[C@H](Cc1ccco1)N1CC[NH+](C2CCCC2)CC1', 'N#C[C@@H]1CNCCN1Cc1cc2c(cc1F)OCO2', 'Cc1c(Br)oc2nc(CCCl)ncc12', 'CN(c1cccnc1)[C@H](C(=O)c1ccc(Cl)c(F)c1)C[C@H]1CCCO1', 'O=C([O-])c1cc(C2CC2)on1', 'Clc1cccc2nn(S[C@@H](c3ccccc3)c3ccc4ccsc4c3)nc12', 'Clc1nnnn1CC[NH2+]Cc1ccccc1OC(F)F', 'Clc1cccc(S[C@H]2OCc3ccccc32)c1[N+](=O)[O-]', 'COc1nnn(C(=O)Cn2ncc3c(O)cc(Br)c(C)c3c2=O)c1C', 'O=C(c1c[nH]c(Nc2cccc(F)c2)n1)N1CCC[C@@H](C(F)(F)F)C1', 'Clc1cccc(-c2cccn2CC[NH2+]C2CC2)c1', 'ClCc1nnsc1Cl', 'ClCc1nnsc1Cl', 'CN=C/[C@@H]1c2ccccc2C(=O)N1c1ncccn1', 'Clc1ccc([C@@H]([C@@H]O)N[C@@H]2CCc3ccccc32)cc1Cl', 'Oc1cc(F)cc(/C=C/C[NH2+]Cc2ccc(F)cc2)c1', 'Clc1ccc(Br)cc1CC[NH2+]C1(c2ccc(F)cc2)CCOCC1', 'Clc1nncn1/n1/nc(CSc2ccc(F)cc2)n2ccnc21', 'Clc1cc(Cl)c(C[NH2+]CC2CCCCC2)cc1F', 'O=[N+]([O-])c1c(O)ccc2c1NC(Cc1ccccc1Cl)C2=O', 'Cc1ccsc1CNC(=O)[C@H]1CCC(=O)NC1=S', 'CNS(=O)(=O)c1ccc(F)cc1N[C@H]1CCC[C@@H](S(=O)(=O)N(C)C)C1', 'Oc1cc(F)nc(CN2CC[C@@H](N3CCO[C@@H](CO)C3)C2)c1', 'ClC(=S)N\\c1ccc(F)c(-c2cscn2)n1', 'O=C([O-])[C@H](Cc1ccc(/N=C/c2cccc(OC(F)F)c2)o1)c1ccccc1', 'O=[N+]([O-])c1cc([C@H]2CC(=O)N(c3ccc(Cl)cc3)C2=O)ccc1[O-]', 'N[C@@H](Sc1nc(-c2ccccc2F)n[nH]1)C1CC1', 'Clc1nncn1-n1cccc1', 'ClC[C@H]1OCC[C@@H]1N1CCc2ccccc2C1', 'Clc1cc(CNC[C@](O)(c2ccccc2)c2ccc(C#N)cc2)cc([O-])n1', 'O=C([C@H]1CCOC2(CCOCC2)C1)N1CCN(C(=O)[C@@H]2C[C@@H]2c2cccc(F)c2)CC1', 'Clc1ccc(-c2cc(-c3cnc([C@H]4CCCO4)nn3)[nH]n2)cc1Cl', 'O=[N+]([O-])c1cc(N[C@@H](CO)c2ccc(F)c(F)c2)cn1C1CCCC1', 'COC(=O)[C@]1(F)CCN(C(=O)c2cc(F)c(F)cc2Br)C1', 'Oc1ccccc1[C@@H]1N[C@@H](c2cc(OCc3ccccc3)ncn2)CCO1', 'Nc1c(/C=N/O[C@H]2CCCc3ccccc32)ncnc1NCc1ccc[nH]1', 'ClCc1cnccc1CCc1nc2c(c(=O)[nH]1)S2', 'O=C(/C=C/O[C@@H]1CCS(=O)(=O)C1)Nc1ccccc1C(=O)N1C[C@@H](C)CC[C@H]1C(=O)NCC(F)(F)F', 'O[C@@H]([C@@H]1CSc2ccc(F)cc21)c1ccc(Br)cc1', 'C=C[C@@H](CC(C)C)[NH2+]C1CCN(C(=O)Nc2ccccc2)CC1', 'O=C([O-])[C@H](Oc1ccccc1F)C(F)(F)C(F)(F)F', 'ClC(=S)O[C@H]1CCCC[C@@H]1OCc1ccccn1', 'Clc1nnc(Cl)nc1F', 'Oc1ccc([C@H](Nc2cc(Cl)ccc2F)N2CCCCC2)s1', 'Clc1cc(C[C@@H]2SCCS2)ncn1', 'O=C(CCOc1ccccc1Cl)[C@@H]1N=NN=C1c1ccc2c(c1)OCO2', 'Clc1ccc(Br)cc1/N=N/SC1=NC=N/c2nccnc21', 'Clc1ccccc1COC(=O)[C@H]1Sc2ccccc2S1(=O)=O', 'Oc1ccccc1[C@H]1OCC[C@@H]1F', 'NS(=O)(=O)NC[C@@]12CCN(c3ccccc3)[C@@H]1Cc1ccccc1-2', 'O=C(/C(c1cccs1)S(=O)(=O)c1ccccc1)[C@@H]1CC(=O)N(c2ccccc2)[C@H]1Cc1ccccc1', 'Oc1cc(F)nc(-n2cnnn2)c1Cl', 'O[C@@H]([C@@H]1C[C@H]1c1ccc(F)cc1)c1ccc(F)cn1', 'Clc1ccsc1S(=O)(=O)N1CC[C@H]2CCCC[C@@H]2C1', 'OC[C@@]1(C(F)(F)F)c2ccccc2N=C(N2CCN(Cc3ccccc3)CC2)c1', 'Oc1cccc2cc(CC[NH2+]Cc3ccncc3)ncc12', 'O=[N+]([O-])c1c([O-])cnn1-c1cccc2c1CCN(Cn1cncc1Cl)C2', 'Clc1nncc(F)c1Br', 'O[C@@H]1c2cc(F)ccc2[C@H]2CCCC[C@@H]12', 'N[C@@H](O)c1cccc(NC(=O)N[C@H](CSC(F)F)c2cccc(F)c2)c1', 'Clc1cc(CCl)n2cncc2n1', 'C[C@@H](c1cccc(C#N)c1)N(C)C(=O)CNC(=O)[O-]', 'O=C(C1=N[C@@H]2[C@H](c3ccccc3)C[C@@](C)(C)[C@H]12)c1ccccc1', 'NS(=O)(=O)/C=c1\\nc(Cl)nc(Nc2ccccc2)c1=O', 'O=[N+]([O-])c1cc(OC[C@H]2CCOC2)nc(-c2nccn2C)c1', 'NS(=O)(=O)Cc1ncsc1CO', 'Clc1cc(NSc2ccncn2)ncc1Cl', 'Clc1ccsc1[C@@H]([NH3+])c1ccccc1', 'Clc1cccc2nn(S[C@@H](Cc3cccnc3)c3ccccc3)nc12', 'Nc1[nH]ncc1SCC(=O)N1C[C@@H](O)C[C@H]1c1ccc(Cl)cc1', 'O=C(/C(Cc1cccnc1)Nc1nc(-c2cccs2)cs1)[C@@H]1COc2ccccc2N1', 'Oc1cc(O[C@@H](c2nc3ccccc3s2)C[C@H]2CCCO2)cc(O)c1', 'Clc1cccc(-c2n[n-]nc2CN2CCCC[C@H]2c2ccccc2)c1', 'Oc1cc(O)c(Cl)cc1/C=C/C(=O)[O-]', 'Clc1ccccc1-c1csc([C@@H](OCC(F)(F)F)c2cc(F)cc(F)c2F)n1', 'COc1nc([O-])cc(C(=O)NCC[C@H](O)c2ccccc2C)c1N', 'Oc1ccc(O[C@@H](CNC(=O)C[C@H]2COCCO2)c2ccco2)cc1Cl', 'O=C(c1ccsc1)[C@@H]1CN(C[C@@H]2CCCO2)CCN1C[C@H]1CCCO1', 'C[C@@H](c1ccoc1)n1c(N)nc2cccnc21', 'Oc1cc(-n2cccc2)ccc1NC1CC[NH+](Cc2ccco2)CC1', 'O=[N+]([O-])c1c(Cl)ncnc1CS[C@@H]1CCc2ccccc21', 'Oc1ccc([N+](=O)C=O)cc1Br', 'Clc1ccccc1O/N=C(/[O-])N(CC(F)(F)F)C[C@@H]1CCCO1', 'C[C@@H](Oc1ccc(F)cc1)C(=O)NC1CC[NH+]([C@@H](C)c2ccccc2)CC1', 'NS(=O)(=O)N[C@H]1C=Cc2ccc(C(=O)N3CCCC3)cc2N1', 'Clc1ccccc1C(C1CC1)[C@H]1CCC[C@@](O)(Cn2cccn2)C1', 'Clc1ccccc1Cn1nnnc1[C@@H]1CNC(=O)N1C[C@H]1CC=CCC1', 'CN(c1ccn([C@H]2CCCCN2C(=O)C(=O)Nc2cccc(Br)c2)c1)C(C)C', 'N=CCn1cccn1', 'Clc1cccc2ccc(C[NH2+]C(c3ccccc3)c3cccs3)nc12', 'Clc1cc(/C=N/N=C\\c2ncn(-c3ccccc3)n2)c[nH]1', 'O=C(C1=NO[C@@H](COc2ccc([N+](=O)[O-])cc2)N1)Nc1ccc(N2C(=O)CCCC2=O)cc1', 'Clc1ccc([C@@H]([O-])N/N=C/c2ccco2)cc1', 'O=[N+]([O-])c1c(Cl)nsc1C(=O)NC1(CC2(O)CCCC2)CC1', 'CC(C)N(Cc1[nH+]ccn1C)S(=O)(=O)c1ccc2c(c1)CCN2C(=O)OC(C)C', 'Clc1ccsc1N[C@@H]1c2ccccc2C[C@@H]1O', 'Clc1cc(NSc2ccc3nccnc3n2)ncn1', 'Oc1cc(-n2nc([C@@H](OC[C@@H]3CCCO3)C2=O)c2ccccc2)ccc1F', 'Oc1cccc(N2C(=O)[C@H]3CN(c4ccc(Br)cc4Cl)CN3C2)c1', 'Clc1ccc(Br)c(/N=C/C=C/C=C2\\C(=O)N(Cc3ccccc3)S/C2=N\\c2ccco2)c1', 'O=C(c1c[nH]nc1-c1ccoc1)N1CC[C@@H](c2ccc(F)c(F)c2)C1', 'Clc1nncc(-c2cn3c(cnn3-c3cccc(F)c3F)c2)c1Cl', 'Clc1cc(F)ccc1-c1cccc(S(=O)(=O)/N=C(\\[O-])CCC[C@@H]2CCCO2)c1', 'Clc1nnn(-c2ncc3nc[nH]c3c2)c1Cl', 'O=[N+]([O-])c1cc(C(=O)N(Cc2ccco2)C[C@H]2CCOC2)on1', 'O=C(c1c[nH]nc1C(F)(F)F)N1CCC[C@@H]1CO', 'O=C(/C=C1/N=NC(=O)N(C2CCC2)C1=O)c1ccc2c(c1)COC2', 'CNC(=O)c1csc(C(=O)O[C@@H]2CC[C@H](Sc3ccccc3)C2)c1', 'O=C(c1c[nH]nc1C[C@H]1CN2CCCC[C@@H]2CO1)NCc1nc2ccccc2c(=O)[nH]1', 'O=C([C@H]1c2ccccc2S(=O)(=O)C[C@@H]1NC(=O)Cc1c[nH]c2ccccc12)N1CCCCC1', 'COCC[NH+](C)CCn1c(-c2ccc(F)cc2)nc2ccccc21', 'O=C(/C(c1cccs1)=N/NC(=O)[C@H]1C[C@@H]1c1cccc([N+](=O)[O-])c1)c1cccc(Cl)c1Cl', 'Oc1cc(O[C@@H](C)C(=O)N2CCC[C@H]2C[C@@H]2CCCO2)ccc1Cl', 'Clc1ccc(-c2nonc2NC[C@@H]2CCCSC2)c(F)c1', 'O=[N+]([O-])c1ccc(C[NH+]2CCC(c3nc4ccccc4o3)CC2)cc1', 'Oc1cc(OC2CC[NH2+]CC2)ccc1Oc1cccc(Br)c1Br', 'O=[N+]([O-])c1c(-c2cccnc2)cnnc1O[C@@H](CN1CCOCC1)c1cccs1', 'ClC[C@H](c1ccco1)n1cncn1', 'Clc1ccc([C@@H](O)[C@@]2(CBr)CCOCC2)cn1', 'O[C@@H](c1cccs1)[C@H]1COc2ccc(F)cc21', 'Clc1cc(Br)c(/C=C/C(=O)[O-])cc1O', 'Clc1ccc(C[NH2+]C2CCSCC2)cc1', 'Oc1cc(F)nc([C@@H]2CCCN2c2ncccn2)c1', 'O=C([C@H](NSCc1cc(F)c(F)c(F)c1)N1CCOCC1)NCC1(CO)CC1', 'O=C([O-])[C@@H](Oc1ccccc1Br)n1cccn1', 'OCc1nccnc1C[C@@H](C[C@@H]1CCOC1)NC(=O)c1cccnc1', 'Oc1cc(-n2cc(I)c([O-])n2)ccc1F', 'Oc1ccc([N+](=O)c2cc(CCO)c(C(=O)[O-])cc2Br)cc1', 'O[C@@H](S[C@@H](Cc1ccccc1)c1ccc(Cl)c(Cl)c1)c1cc(Cl)ccn1', 'Oc1cc(-n2nc([O-])[nH]c2=O)c(=O)n(Cc2cccc(Cl)c2)c1', 'O=C(/N=C/NC(=S)N1CCCC1)Sc1cccc(I)c1', 'ClCc1noc(C[NH2+]Cc2cnn(Cc3ccccc3)n2)n1', 'Oc1cc(O)cc2c1C[C@H](CNCc1n[nH]c(C3CC3)n1)C2', 'O=[N+]([O-])c1c(Cl)n2nnc(C3CCCCC3)c2[nH]1', 'Clc1ccc(-c2cccnc2Sc2ccc(C[NH+]3CCCC3)cc2)cc1', 'Clc1ccccc1-c1cscc1C[NH2+][C@H]1CCc2ccccc21', 'OC[C@@H](c1ccco1)N1CC[C@@H](CS(=O)(=O)c2cccc(F)c2)C1', 'Oc1ccc([N+](=O)c2occc2Cl)cc1O', 'OC[C@@]1(NCC2CC2)CCC[C@@H]1CCOc1cccnc1', 'Oc1ccccc1CC[C@H](O)C[NH2+]Cc1cc(-c2ccccc2)no1', 'Clc1nncn1c1c(Br)cc(I)cc1O', 'ClC([O-])=NS(=O)(=O)c1c(F)cc(Cl)cc1Cl', 'O=[N+]([O-])c1c(-c2nsc([C@@H]3CCOC3)n2)cnn1Cc1ccccc1', 'O=[N+]([O-])c1cc(N[C@H]2CCc3c(Br)cccc32)ncn1', 'Clc1nnnc(Cc2ccc(Br)s2)c1N', 'Cc1cn(-c2ccccc2)c(C(=O)NC[C@H](c2ccccc2)[NH+]2CCCCC2)c1', 'ClCc1cn(Cc2ccc(F)cc2)nc1S[C@@H]1CCSC1', 'Clc1ccc([C@@H]([C@@H](O)[C@H]2CCCO2)Sc2ccccc2)nc1', 'Clc1ccccc1O/N=C1\\SC(=S)N(CC[NH2+]C2CCCCC2)[C@H]1c1cccc(Br)c1', 'Clc1ccc(Br)cc1O[C@H](CO)C(=O)[O-]', 'Clc1ccc(-c2csc3[nH]nnc23)s1', 'Clc1ccc(-c2csc3[nH]nnc23)s1', 'O=[N+]([O-])c1c(Br)cccc1[O-]', 'Oc1ccc([N+](=O)C=C(Cl)Cl)cc1SCCO', 'O=C(c1cn[nH]n1)N(c1c(Cl)cccc1Cl)N1CCSCC1', 'O=C(c1c[nH]cn1)O[C@H]1CC(=O)N(C2CC2)C1', 'Clc1ccc([C@@H]([NH2+][C@@H](CO)c2ccccc2)C2CC2)nc1', 'ClC(F)(F)C1CCN(C(=O)[C@@H]2CCCOC2)CC1', 'O=C(/C(c1cccs1)(C(=O)[O-])C1CCN(S(=O)(=O)c2ccc([N+](=O)[O-])cc2)CC1)/c1ccccc1', 'Oc1ccccc1OC1CC(C(F)(F)F)O[C@@H]1c1ccccc1', 'Oc1cccc2cc(OC[C@@H](O)[C@@H](c3ccccc3)C(F)(F)F)cnc12', 'Clc1cc(Br)c(C2=NC[C@@H]2CN2CCO[C@H](CCO)C2)cc1Cl', 'OCS/C(=C/C=C/O)C1(c2ccc([N+](=O)[O-])cc2)CC1', 'Clc1nncc(Cl)c1I', 'OCc1nncc(-c2ccco2)c1C1CC[NH+](Cc2ccccc2)CC1', 'Nc1nnnn1CC[NH+]1CCN(Cc2ccccc2)CC1', 'Oc1cc(F)nc(SCC(F)F)n1', 'Clc1ccc([C@@H](Cl)[C@]2(F)CCc3ccccc32)cc1', 'Clc1cc(F)c([C@H](Cn2cc(Br)cn2)N(C2CCCC2)S(C)(=O)=O)cc1', 'CCCCn1ncc2c1N=C(C)O[C@@H]2c1ccccc1C', 'Clc1nncc(F)c1C(F)(F)CC(F)(F)F', 'O=[N+]([O-])c1c(Br)scc1N1CCn2c1nn2-c1ccc(Br)cc1', 'Clc1ccc(Br)cc1S(=O)(F)c1cnn(Cc2ccccc2)c1', 'OC[C@H]([NH2+]Cc1ccccc1)C1CC1', 'ClCc1csc2nc[nH]c12', 'Nc1cn[nH]c1NC(=O)[C@H]1Cc2ccccc2S1', 'Clc1cc(C(CO)OCc2ccncc2)n[nH]1', 'O=C([C@H](Cl)Cc1ccccn1)[C@@H]1CCC[C@@H](O)C1', 'Clc1ccc(-c2cc[nH]n2)nc1O', 'OC1([C@H]([O-])c2ccc(-n3cccn3)cc2)CCOCC1', 'Clc1nnnn1-c1cnn(C[C@@H]2CCCO2)c1', 'N=CCNc1c(-c2[nH+]c3ccccc3[nH]2)c(=O)oc2ccccc12', 'O=[N+]([O-])c1cc(Br)ccc1Oc1ccccc1C[NH2+]C[C@@H]1CCCO1', 'Clc1ccc(C(CCBr)COCCO)s1', 'Oc1ccccc1CN1CC[NH+]([C@H]2CCOc3ccccc32)CC1', 'O=C([C@H]1[C@H]2C(=O)N(CCOCC(F)(F)F)C(=O)[C@H]12)c1cccs1', 'Clc1nnc(Br)cn1', 'Oc1cc(-n2cc(C(F)(F)F)ccc2=O)ccc1NC[C@H]1CCCC[C@@H]1O', 'ClC[C@H]1CCS[C@@H]1c1cccc(Cl)c1', 'O=C([C@H]1[C@H]2CCCC[C@@H]2C1=O)Nc1ccc(I)cc1', 'Clc1ccccc1CCn1cc[nH+]c(C[NH2+][C@H](CO)c2ccccc2C(F)(F)F)c1=O', 'Clc1ccsc1-c1nnc(C[C@@H]2C[C@H]2c2ccccc2F)o1', 'Cc1ccsc1NC(=O)C1CCN(C(=O)[C@@H]2CCO[C@@H]2C)CC1', 'Clc1ccc(Br)cc1N(C(=O)c1ccc(F)cc1)[C@@H]1CSCS1', 'Clc1ccc(CNC[C@@H]2CC[C@@H](C(F)(F)F)O2)cc1[N+](=O)[O-]', 'Oc1ccc(O[C@@H]([NH3+])C[C@@H]2OCc3ccccc32)cc1', 'Clc1cc(Cl)c(C2CC2)cc1N[C@@H]1CC[C@H](S(=O)(=O)/N=C/c2ccc3c(c2)OCO3)C1', 'Clc1nnnc(Cl)n1', 'Clc1nnnc(Cl)n1', 'O=[N+]([O-])c1c(Br)cccc1-n1cc([O-])cn1', 'O=C(COc1cc(Br)ccc1O)NCC(=O)Nc1ccc2n[nH]ccn2c1', 'Clc1ccccc1-c1n[nH]c2cncc(NCC[NH+]3CCCCC3)c12', 'O=C(c1cccs1)C1=CC(c2cccc(Cl)c2)SC(=S)N1c1ccccc1', 'Oc1ccc([C@H](N[C@@H](Cn2cccn2)C2CC2)N2CCCC2=O)cc1', 'O=C([O-])c1cnc(S[C@@H]2CCS(=O)(=O)C2)n1-c1ccc(Cl)cc1', 'O=[N+]([O-])c1c(OCO)nc(-c2cccnc2)n1C[C@@H]1CCCO1', 'Cc1c(-n2cccc2)c2ccccc2n1CC(=O)N[C@H]1CCC[C@@H]1[S@@](=O)Cc1ccccc1', 'ClCc1cn[nH]c1SCC[S@](=O)Cc1ccccc1', 'Nc1nnnn1CC[S@@](=O)c1ccccc1Cl', 'Clc1ccc(C(N)=NC2CCCS2)cc1', 'O=C(/C(c1cccs1)SC1=NC=N1)c1ccc(Cl)cc1', 'OC[C@@]1(Nc2cccc([N+](=O)[O-])n2)CCCc2ccccc21', 'Clc1cccc([S@@](=O)[C@@H]2CC[C@H](Nc3ccccc3F)C2)c1', 'O=[N+]([O-])c1ccncc1[C@@H]1OCCN(c2ccc(OS(=O)(=O)c3ccccc3)nc2C2CC2)C1', 'Nc1[nH]ncc1C[S@](=O)Cc1ccccn1', 'O[C@@H](c1cc(F)cc(F)c1)c1nc(C2CC2)ns1', 'Oc1cccc2ncn(/C(CO)c3nc(-c4ccccc4F)n[nH]3)c12', 'Clc1ccccc1Oc1cc([N+](=O)n2cc(Br)cn2)c[nH]1', 'N#C[C@@H](c1ccccc1)[C@H]1CCCN1C(=O)N[C@H](c1cccc(C)c1)c1ccccc1', 'ClC[C@@H]1C[C@@H]1c1cc(F)cc(F)c1', 'Clc1nncnc1[N-]N1CCN(C(=O)C2CCCCC2)CC1', 'O=[N+]([O-])c1ccc2ncn([C@@H](c3ncccc3Cl)C(F)(F)F)c2c1', 'Clc1ccc([C@@H]([O-])N2C[C@@H](CNc3ncccc3C#N)CC2)cc1', 'N=C1\\[C@@H](c2cccc(F)c2)NC(=S)NC1=O', 'Clc1ccccc1Nc1cc([C@H]2CS(=O)(=O)CCN2)ncn1', 'Clc1ccsc1[C@H](N[C@@H]1CCSC1)c1ccccc1', 'O=C(C1CC[NH+]([C@@H]2CCN(C(=O)c3ccc4c(c3)OCCO4)CC2)[C@@H]1O)c1ccccc1', 'O=[N+]([O-])c1c([O-])sc(OC2CCCCC2)c1[N+](=O)[O-]', 'N#C/C(=C\\c1cn(CC(=O)[O-])c(C(=O)N2CCOCC2)c1C#N)c1ccco1', 'Clc1ccccc1NC1CC[NH+](C2CCCC2)CC1', 'Clc1ccc(CNc2c[nH]c(C(=O)[O-])c2)o1', 'Clc1ccc(Br)cc1N1CCN(CC2[NH+]=c3ccccc3=[NH+]2)CC1', 'Clc1ccc([C@@H](OC2CCCC2)C(F)(F)F)o1', 'O=C([C@H](c1ccc(Cl)cc1)N1CC[NH2+]CC1)NCCOC1CC1', 'OC[C@H](c1ccccc1)[NH+]1CCN(c2ccccc2)CC1', 'Clc1ccc([C@@H](Br)[C@H]2CSCCO2)cc1', 'Clc1cccc(Br)c1NC[C@]1(O)CCSC1', 'C=CCc1ccccc1OCc1nnc([S@](=O)CC(C)(C)CC)n1C', 'Clc1ccc(Br)cc1S(=O)(c1ccccc1)c1ccccc1', 'Clc1ccccc1Cc1nc(Cc2[nH+]cc(Cl)cc2Cl)n[nH]1', 'Nc1cnnn1Cc1cc(Br)ccc1O[C@H]1CCOC1=O', 'OCc1n[nH]c([C@@H](C#N)c2ccc(Cl)cc2)c1Cl', 'Oc1cccc(O[C@@H]2CCO[C@]2(Cc2ccccc2Br)C2CC2)c1', 'Cc1ncsc1CO[C@@H]1CCC[C@@H](C)C1', 'CN(CC(F)(F)c1ccc(Cl)cc1Cl)C(=O)N[C@@H]1CCC[C@H](C)C1', 'O=C(Cc1c[nH]nc1-c1ccccc1)[C@H]1CCC[NH+](Cc2c[nH]c3ccccc23)C1', 'Clc1ccccc1C[NH+](C1CC1)[C@H]1CCCc2ccccc21', 'O=C(CC1(C2=Nc3ncnn23)[C@@H](CO)C1)N1CCCCC1', 'O=[N+]([O-])c1c([N+](=O)[O-])cnn1[C@@H]1CCS(=O)(=O)C1', 'CN(CC1CCC([NH+]2CCCCC2)CC1)C(=O)NCCCn1cncn1', 'ClC[C@H](c1nc2ccccc2s1)N1CCO[C@@H]2CCCC[C@H]21', 'O=C(/C(c1cccs1)[NH+](C)Cc1ccc(Cl)cc1)C1(N2CCOCC2)CCCC1', 'O=C([C@H](SC1CCCCC1)N1CCCC1)OC1CCCCC1', 'Clc1nnnn1/C(Cl)Cc1cccc(Br)c1', 'O=C(C[NH+]1CCO[C@H](c2ccccc2)C1)Nc1ccc(Cl)cc1', 'CCNC(=O)N[C@H](C)c1nc2ccccc2n1CCN1CC[NH+](C)CC1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1[C@H]1CCS(=O)(=O)C1', 'O=C(c1cn2ccsc2n1)[C@@H]1CCCCO1', 'ClCc1csc(O[C@@H]2CCOC2)n1', 'Clc1ccc(Br)cc1O[C@@H]([NH3+])C1CCCC1', 'O=C(CNC(=O)Nc1nc2c(s1)CC[C@@H](C(=O)[O-])O2)c1ccccc1', 'Clc1ccc(Br)cc1SC/C=C\\c1cnc(Oc2ccccc2)c([O-])c1C(=O)[O-]', 'Clc1ccc(Br)c(NC(=S)N(c2nccc([N+](=O)[O-])c2)[C@H]2CCSC2)c1', 'Clc1ccc(Br)cc1[C@@H]1O[C@H](CO)[C@H](O)[C@H]1Cc1ccccc1', 'Clc1ccsc1S(=O)(=O)[N-]c1cccc(C(=O)NOc2ccccc2)c1', 'O=C(/N=C\\NCCc1cccc(Cl)c1)c1nc([O-])c2ccc(Cl)cn12', 'Clc1cccc(Br)c1CC1OCCC[C@@H]1C(=O)[O-]', 'Oc1ccc([N+](=O)Cc2n[nH]c(Cn3c(=O)[nH]c(=O)n3C)c2C(F)(F)F)cc1', 'O=C(Cc1n[nH]cc1C(=O)[O-])Nc1cc(C(F)(F)F)cc([N+](=O)[O-])n1', 'O=C(/C(c1cccs1)[C@@H]1COCCN1c1cccs1)NCc1ccncc1', 'Clc1ccc(-c2nonc2[S-])nc1', 'Clc1ccc(-c2nonc2[S-])nc1', 'Clc1ccc([C@@H](N[C@@H](O)CO)C(F)(F)F)cc1', 'Nc1[nH]ncc1CN(C1CCCCC1)[C@H]1CCSC1', 'O=[N+]([O-])c1c([O-])nsc1Nc1cccc(I)c1', 'ClC[C@@H]1C(=O)N2CCCNC2=C1C(=O)c1cccc(F)c1', 'O=C(c1ccc[nH]c1=O)[C@@H]1CCC[NH+](Cc2ccc(Cl)cc2)C1', 'Clc1cc(/C=C/C[NH+](Cc2ccco2)Cc2ccco2)n(C2CC2)c1', 'Clc1cc(CCl)n[nH]1', 'Clc1cc(CCl)n[nH]1', 'Clc1ccc(COc2nc(N3CCCCC3)[nH]c2N2CC[C@H](C[NH+]3CCCC3)C2)cc1', 'Clc1ccc(Br)c(C[NH2+]C[C@@H]2CCN(c3cccc(F)c3)C2)c1', 'Clc1ccsc1/CCc1cnc(-c2cccs2)n1C1(C#N)CCCC1', 'N#C[C@@H]1C(=O)N(CCC(=O)Nc2cc(F)ccc2F)C(=O)N(C(C)(C)C)C1=O', 'OC[C@@]1(CNCc2cc3ccc(F)cc3n2C2CC2)CCCO1', 'COc1ncc(Cl)c([C@@]2(CC)COc3ccccc32)c1[N+](=O)[O-]', 'Clc1ccc([C@@H]([O-])Cn2cncn2)c(Cl)c1', 'O=C([C@H](Cl)Br)[C@@H](C)c1ccc(Cl)cc1Cl', 'O=C([C@H](Sc1n[nH]c2ccccc12)c1cc2c(cc1F)OCCO2)N1CCSCC1', 'Clc1nnn(C[C@@H](Br)CO)c1-c1ccc(Cl)cc1F', 'Oc1ccc(O[C@@H]2C[C@@H]2C(=O)O[C@H]2CCCc3ccccc32)cc1[N+](=O)[O-]', 'N[C@@H]1NC(=O)/C1=C/c1ccc(-c2ccc(F)cc2Cl)o1', 'COc1nnnn1[C@@H]1CC(C)(C)CCN1C(=O)c1cc(F)ccc1F', 'Clc1nncc(F)c1N[C@H]1CCc2c(Br)cccc21', 'Clc1nnnn1/C([O-])=N/c1ccc([N+](=O)[O-])cc1', 'Clc1ccccc1OC1CC[NH2+]CC1', 'Oc1cccc(Br)c1C[NH2+][C@@H]1CCCc2ccccc21', 'O=C(/C=C\\C[N-]c1ccc(Cl)cc1Cl)C1CCCCC1', 'Clc1ccc(Cc2nccc(CC[NH+]3CCOCC3)n2)cc1', 'Clc1ccc(C[NH2+][C@@H]2CCCC[C@H]2CN2CCOCC2)cc1', 'Clc1nnnn1CCCC[NH+]1CCC(Cc2ccccc2)CC1', 'Oc1ccc(O[C@@H]2CC[NH2+]C[C@H]2Cc2ccccc2)cc1', 'Clc1cc(/C=C/NC2CC[NH+](Cc3ccccc3)CC2)ccc1F', 'NS(=O)(=O)CC[C@@H]([NH2+]Cc1ccccc1)c1ccccn1', 'COc1nnnc(N2CCc3nn(-c4ccc(F)cc4)n(C)c3c2C(=O)OC)c1C', 'N#C[C@@]1(Sc2ccccc2)NC(=O)C=C1c1ccccc1', 'Clc1cccc2nc([C@@H](CO)NNC(=O)c3cccc(Cl)c3)nc([O-])c12', 'Clc1ccccc1O/N=C(/[O-])C[C@H]1Cc2ccccc2S1', 'Clc1cccc(S[C@@H]2OCCNC2=O)c1', 'CN=C/C(=N\\OC[C@@H](O)c1ccc(Br)cc1)C1CC1', 'Clc1ccsc1Sc1nc(CSc2ccncc2)c[nH]1', 'Clc1cccc2[nH]nc(C[C@@H](C(=O)[O-])N3CCCC3c3ccccc3)c12', 'O=C(CCNC(=O)N1CCOCC1)N1C[C@@H]2CCCN[C@@H]2C1', 'Oc1ccc(F)c(F)c1SCCN1CC[NH+](Cc2ccccc2F)CC1', 'Clc1cccc(F)c1N1CCC(N/C=C2\\C3=C(CCO3)CC2)C1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccn1[C@H]1CCOc2ccc(F)cc21', 'COC(=O)Nc1ccc(O[C@@H](C(C)C)[NH+](C)C)cc1', 'ClCc1cncc2[nH]nnc12', 'Clc1cccc(NCC[NH2+]Cc2sccc2Cl)c1', 'Oc1ccc([N+](=O)c2occc2Cl)c2c1OCCO2', 'Clc1ccccc1[C@H]([NH2+][C@H]1CCc2cccnc21)c1ccccc1', 'Clc1ccc(C2(Sc3ncnc4ccccc34)CCC[C@@H]2CO)cc1', 'Oc1cc(O)c(/C=C/C(=O)[O-])cc1Br', 'Clc1ccccc1[C@H]([NH3+])[C@H]1CSC(=O)N1Cc1ccccc1', 'Clc1ccc([C@@H]([O-])[C@@H](c2ccccc2)N2CC[NH+](CC(=O)NC3CCCC3)CC2)cc1', 'O[C@@H](S[C@@H](N)c1cccc([N+](=O)[O-])c1)S(=O)(=O)c1ccccc1', 'Cc1ncsc1C(=O)N1CCN(C[C@H](O)C(C)(C)CCC#N)CC1', 'Clc1cc(N2CCC[C@@](O)(C(F)(F)F)C2)ncc1F', 'N=C1/C(=C/c2ccc(OCc3cc(C(=O)NCc4cc[nH+]cc4)ccc3)cc2)C(=O)SC1', 'Clc1cc(C[NH+](Cc2ccco2)Cc2ccccc2)ccc1Cl', 'Clc1cccc2cc(C[NH2+][C@H](CO)c3ccccc3)c(=O)[nH]c12', 'Oc1cc(-n2nnnc2C2CC2)c[nH]1', 'Cc1cnc2c(c1)[nH]c(=N)n2C[C@@H]1CCCO1', 'Oc1cc(O)cc(Cl)c1C[NH2+]Cc1ccc(Cl)nc1', 'ClC[C@@H]1CCc2cc(Oc3ccc(Br)cc3)oc21', 'Clc1ccsc1[C@H](CSc1nnc(Cc2ccccc2)n1C(F)F)N1CCCC1', 'ClC(=O)c1ccc(N[C@@H]2CSCCS2)cc1', 'O=C([C@H](Cl)c1[nH]ccc1C(F)(F)F)[C@@H]1CO[C@@H]1c1ccccc1', 'Clc1ccc(Br)c([C@@H]2SCCC[C@@H]2N2CCOCC2)c1', 'Clc1cccc([S@@](=O)c2ccc(Br)o2)c1', 'Clc1ccc([C@@H](Br)[C@H]2CC[C@@H]3CCC[C@H]23)cc1', 'Clc1cccc2nn(S[C@@H](CO)c3ccccc3)nc12', 'Nc1n[nH]nc1CN(Cc1ccco1)[C@@H]1CCS(=O)(=O)C1', 'CCCC(CCC(=O)[O-])C(=O)N1CCC(C)CC1', 'Clc1ccc(-c2cc([C@H]([NH3+])C3CCCC3)cnc2C#N)cc1', 'COC(CNC(=O)N1CC[C@H](C)[C@@H](O)C1)c1cccc(F)c1', 'COc1nnnc(N2CC[C@@H](Cc3csc(C(=O)NCCCOC)n3)C2)c1C', 'Oc1ccc([N+](=O)c2onnc2Cn2cncn2)cc1', 'COc1ncncc1C[C@@H]([NH3+])Cc1cc(Cl)ccc1Cl', 'ClCc1nc(-c2ccc([O-])cn2)cs1', 'O=[N+]([O-])c1cc([C@H]2SCC(=O)N(Cc3cccs3)C2=O)ccc1OS(=O)c1ccccc1Cl', 'COc1nn[nH]c1/C=C/C(=O)N(CC#N)C1CC1', 'Cc1noc(CCc2c(C#N)c([O-])nn2-c2cccc(C)c2)c1C', 'Nc1nnnn2c(C(=O)[O-])ccc12', 'N#Cc1c(Br)cccc1N[C@H]1CCSC1=O', 'O=C(Cc1cc(F)ccc1O)NNC[C@@H](O)c1cscc1F', 'O=C(Cc1c([O-])c2cnccc2oc1=O)NNC(=O)Cn1ccc(=O)[nH]1', 'Clc1ccccc1NC(=S)N1/C(=N/N=C\\c2ccc(C(F)(F)F)s2)SC1=S', 'Clc1cccc([S@@]C(Br)(CO)C(F)(F)F)c1', 'O=C(CS(=O)(=O)C[C@H]1CCCCO1)NN1CCOCC1', 'O=C(c1cn2ccsc2n1)[C@@H]1C[C@H]1c1ccccc1F', 'Clc1ccccc1Oc1cc(O[C@@H]2CCOC3(CCOCC3)C2)nc2ccccc12', 'ClCc1nc([C@@H]2CCCN(C(=O)[C@@H]3CCCO3)C2)cs1', 'Oc1ccccc1[C@H]1N=NC[C@@H]1c1nc(-c2ccc(Cl)cc2)no1', 'Clc1ccc(Br)c(S[C@H]2CCOC3(CCCCC3)C2)c1', 'ClCc1nnc(CNS(=O)(=O)[C@H]2CCCc3ccccc32)s1', 'Clc1nnnn1/C(/[O-])CC(=O)Nc1cccc(C(F)(F)F)c1', 'Nc1cn[nH]c(=O)c1[N-]S(=O)(=O)c1ccc([N+](=O)[O-])cc1', 'ClCc1n[nH]c(O)c1C(F)(F)F', 'O=C([O-])[C@H]([C@H](O)Cc1ccc(CC(F)F)cc1)N1CCOCC1', 'Clc1ccc(-c2nonc2)c(C(=O)[O-])c1', 'OC1=C[C@@H](C(=O)Nc2ccccc2O)[C@@H]1C', 'ClC(F)/C=C/C=C/C=C/C=C/c1ccc(Cl)c(-c2nc(C3CC3)cs2)c1', 'O=[N+]([O-])c1ccnc2c1OCC[C@H]2NC(=O)[C@H]1CCCc2ccccc21', 'O[C@@H]([C@@H]([NH2+]Cc1ccccc1)Sc1ccccc1)c1cccc(F)c1', 'O[C@@H]1C[C@@H](c2cccs2)N(c2ccc(Cl)cc2)C(=O)O1', 'Clc1cc(F)c(/C=N\\NC(=O)[C@H]2CSCCS2)cc1Cl', 'Nc1[nH]nc2c1[C@@H](c1ccc(Cl)cc1)C1=C(CCCC1=O)C2', 'Clc1ccc([C@@H](O)[C@]2(O)CCc3cccnc32)cc1', 'Oc1ccc(O[C@H](C[C@@H]2CCCCO2)C(=O)c2ccccc2)c2c1C(=O)N=C(c1ccccc1)[NH+]2', 'O=[N+]([O-])c1ccn(CC[C@H]2CCC[C@@H]2O)c1', 'Clc1ccc(C[NH2+][C@@H](CO)c2ccco2)cc1', 'Clc1ccccc1-c1csc(C[NH+]2CCCCC2)n1', 'Oc1ccc(O[C@H](C[NH+]2CCCC2)c2ccccc2)cc1', 'O=C(COc1ccc2nnnn2c1)NCc1cccc(C[NH+]2CCCC2)c1', 'Cc1snnc1S[C@@H](C)C(=O)N[C@@H](c1ccc(F)cc1)C1CC1', 'Clc1ccccc1Oc1cnc2nn[n-]c2c1', 'O=C([O-])[C@@H]1CC(c2ccccc2)=Nc2ncnn21', 'Clc1ccccc1N1N=C2SCC[C@@H]2C1=O', 'Clc1cc(/C=C(/[O-])/N=[N+]=NC(=O)c2ccccc2)cc(Br)c1O', 'CN=C(OC)CC[NH+]1CCN(CCc2ccc(Cl)cc2)CC1', 'Clc1ccccc1-c1csc([C@H]2CCCC[C@@H]2C(=O)[O-])n1', 'Clc1cccc(F)c1C[S@@](=O)[C@@H]1CC(=O)N(Cc2ccc([N+](=O)[O-])cc2)C1', 'Clc1ccc(CCC[NH2+][C@@H]2CCCc3cc(OCc4ccc(Cl)cc4)sc32)cc1', 'Clc1nnn(C(C)(C)C2CCSCC2)c1N1CCCC1', 'O[C@@H](Sc1nnns1)C(=O)Nc1ccc(Cl)cc1', 'Clc1ccc(Br)c(OC2CC[NH+](C3CCCC3)CC2)c1', 'Clc1cccc(Br)c1C[N+]1=CCCC1', 'Clc1cc(C[C@@H]2Cc3ccccc3S2)[nH]c1NCc1cc(F)ccc1F', 'O=[N+]([O-])c1ccc[nH+]c1Nc1ccc(/C=C/C(=O)[O-])cc1', 'Clc1cc(NSc2ccn[nH]2)ccc1N/N=C/Oc1cccnc1', 'O=C([O-])/C=C/Sc1cccc(F)c1[N+](=O)[O-]', 'COc1nccc([C@@H]2N=NC(=O)c3ccc(OC(C)C)cc32)c1C', 'Oc1ccccc1/N=C1/C(=C/c2ccc3c(c2)OCO3)N=C2CCC[C@@H]21', 'CCO[P@](C)(=O)c1ccc(Cl)c([N+](=O)[O-])c1', 'Clc1cc(NSc2nncs2)ccn1', 'O=C([C@H](c1c(Cl)cccc1Cl)N1CCCC1)N(C1CCCCC1)S(C)(=O)[O-]', 'Clc1cccc([S@@](=O)CC(F)F)c1O', 'Clc1ccc([C@@H]([O-])N(C[C@H]2CCOC2)C2CC2)cc1', 'Clc1ccc(CN[C@@H]2CCC[NH2+][C@H]2c2ccc(F)cc2)cc1', 'O[C@@H](c1csnc1)c1cc(F)c(F)cc1Cl', 'Clc1nnnc(N2CCOCC2)c1NCC1OCOC1', 'Clc1cc(NCc2ccc(C[NH+]3CCCC3)cc2)ncn1', 'O=C([O-])/C(=N/n1cccn1)Nc1ccc(Br)cc1C', 'Clc1cccc2[nH]nc(S(=O)(=O)[N-]c3ccc(-n4cccn4)c([N+](=O)[O-])c3)c12', 'OCS[C@@H](Cc1ccccc1Cl)c1c(F)cnc(F)c1F', 'Clc1ccc(C(F)(F)F)c(-[n+]2c(C(F)(F)F)nc3ccccn32)c1', 'O=C([C@H](N[C@@H]1CCS(=O)(=O)C1)N1CCCC1)Nc1nccs1', 'ClC[C@H]1O[C@@](NC(=O)C2CCC2)Cc2ccc(Br)cc21', 'COC[C@]1(C)CCN(C(=O)c2cc(C)nn2C2CCOCC2)C1', 'Clc1ccc(-c2cc(O[C@H](C(=O)[O-])C3CC3)[nH]n2)cc1', 'Clc1cccc(/C(Cl)/N=C/C=C\\c2ccco2)c1', 'Oc1ccc(F)c(C[NH+]2CCC(Nc3ccc([N+](=O)[O-])cc3)CC2)c1', 'Clc1cc(Cl)nnc1CC[NH2+]Cc1cccc(C(F)(F)F)c1', 'O=C(COc1ccccc1F)NCC[NH+]1CCc2ccccc21', 'Clc1ccc(C[NH+](CCO)Cc2cccnc2)cc1', 'O=C(c1c[nH]c2nc[nH]c12)N1CCC2(CC1)N(c1ccccc1)CC[C@H]2c1ccccc1', 'O=[N+]([O-])c1c(Cl)nnc2c1COC2', 'Clc1ccccc1O/N=[N+](\\[O-])c1cccc(S(=O)(=O)N2CCCC[C@H]2C(F)(F)F)c1', 'Clc1cc(C[C@@H]2Cc3ccc(Cl)cc3O2)n[nH]1', 'O=C([O-])[C@H](NS(=O)(=O)c1ccc(Cl)cc1)c1cscc1C(F)(F)F', 'Clc1ccc(CO)c([C@H]2CSCCO2)c1', 'Clc1ccc(-c2n[nH]c(N/N=C(\\[O-])c3ncns3)n2)cc1', 'Clc1cccc([S@@]CC[NH+]2CCCC2)c1', 'Clc1ccccc1CS(=O)(=O)CC1CC[NH2+]CC1', 'Clc1nnn(Cc2ccn[nH]2)n1', 'Clc1nnnn1-c1cn[nH]c1', 'OCS[C@@H]1CCCc2[nH]nc(-c3ccccc3)c(=O)c21', 'O=C(c1cn[nH]n1)N(c1nncs1)c1ncccn1', 'Oc1cc(COCOC2[C@@H](N3CCOCC3)CCC2)ccn1', 'NS(=O)(=O)N[C@H]1c2ccccc2C[C@@H]1OC(=O)/C(C#N)=C/c1ccc(Cl)cc1', 'Clc1ccc(-c2csc3[nH]c([C@@H]4CCCNC4=O)nc23)cc1', 'Clc1ccc([C@@H](CC[C@H]2CCOC2)NN=C2CCCOC2)cc1', 'N#C[C@@H]1[C@H](c2cccc(Br)c2)[C@@H](c2ccccc2)OC1=O', 'Oc1ccc(/C=N\\N[C@@H]2CCC[C@@H]2C[C@@H]2CCOC2)cc1', 'OCc1nn(SC[C@@H]2CCCO2)c2ccccc12', 'Clc1cc(F)c(C[S@](=O)CS(=O)(=O)N2CCCC[C@@H]2CO)cc1', 'Clc1ccsc1[C@H]1Oc2ccccc2[C@@H]1Nc1cccc([N+](=O)[O-])c1', 'Oc1ccc([N+](=O)c2ocnc2)c2[nH]ccc12', 'COc1nn[nH]c1CC(=O)N1CC[C@@H](C(=O)[O-])[C@@H]1C', 'O=C([O-])/C=C(/N=C\\c1[nH]nc(C(F)(F)F)c1Cl)NCc1ccccc1', 'ClC[C@@H]1OCCCN1C[C@@H](O)c1ccc2c(c1)CCCC2', 'Oc1cccc([C@H]2OC(/C=N/OC(=O)c3ccccc3)c3ccccc32)c1', 'Clc1cc(F)c(I)s1', 'OCSC1=N[C@@H](COc2cccc(Cl)c2)N(C(=O)c2ccc([N+](=O)[O-])cc2)C1', 'O=C([O-])[C@@H]1CC[C@@H](CNC(=O)Nc2ccccc2O)N1', 'Clc1nncc(N[C@@H]2CCC[C@H](O)C2)n1', 'O=C(COCCC[NH+]1CCCC1)Nc1cc2c(cc1Cl)OCCO2', 'Cc1cnc([C@@H]2[C@@H](c3ccccc3)OC[C@@H]2NC(=O)/C=C/c2ccccc2)s1', 'Clc1nncc(OC[C@@H]2Cc3cc(F)cc(F)c3O2)n1', 'Oc1cc(O[C@@H](C2CCCC2)N2CCCC2=O)cc(Br)c1Cl', 'Clc1cccc([S@@](=O)c2nnc(N3CCCC3)o2)c1', 'Clc1ccccc1C(C)CCC[NH+]1CCCCC1', 'Clc1cc(Cl)c2c3c(ccc2n1)C[C@](O)(Cc1ccc(F)cc1)N3', 'Nc1c(O[C@@H]2CCCc3ccccc32)ncnc1NC(=O)[C@@H]1C[C@@H]2c3ccccc3C(=O)N12', 'O=[N+]([O-])c1ccc2[nH]c(C[NH2+]Cc3ccnc(F)c3)nc2c1', 'Clc1cccc2c(N[C@@H]([C@@H]3CCCO3)[C@H]3CCS(=O)(=O)C3)cccc12', 'Clc1ccc(Br)cc1CC[NH2+]C1(CO)CCCCC1', 'Clc1cccc2c1CCC[C@H]2[NH2+]Cc1ccc(OC(F)F)cc1', 'O[C@@H](c1cscc1C(F)(F)F)C1CC1', 'Clc1ccsc1C[N-]Oc1ccc2c(c1)CCC2', 'Clc1ccccc1C[NH+](Cc1ccccc1)CC1OCCO1', 'O=C([C@H](Sc1ccccc1F)N1CCC(c2ccccc2)(C(=O)[O-])CC1)n1cccc1', 'Clc1nncc(NC[C@@H]2COc3c(F)cccc32)n1', 'CNC(=O)C[NH+]1CCN(c2ccccc2Br)CC1', 'Clc1ccc(C2SCCOC2)nc1', 'Clc1nncn1[C@@H](CO)c1ncccc1F', 'Clc1ccc(C(N)=O)cc1OCc1nnn[n-]1', 'O[C@@H]1c2cc(S(=O)(=O)[O-])ccc2[C@H]1c1ccccc1', 'Clc1ccccc1-c1n[nH]cc1C[NH2+][C@@H]1CCc2ccccc21', 'Clc1ccc(-c2n[nH]c(CO[C@@H]3CC[C@@H]4CCCC[C@@H]4C3)c2[N+](=O)[O-])cc1Cl', 'NS(=O)(=O)/N=C/C1CCCC1', 'Clc1nnc(Br)s1', 'Clc1cc(F)c(Cl)nc1NC[C@@H]1CCCN1C(F)(F)F', 'C=C[C@@H]([NH3+])c1ccc(OC[C@@H]2CCCO2)cc1', 'Clc1ccccc1Nc1nn[n-]n1', 'N#C[C@@H](O)C[C@@H]1COCCN1Cc1ccc(Br)cc1', 'OC[C@@]1(OC2CC2)CCC[C@@H]1c1ccccc1F', 'Nc1[nH]nc(NNC(=O)[C@@H]2COc3ccc(Br)cc32)n1', 'O=C(COc1cc(C2[C@H](C(F)(F)F)N=N2)[nH]n1)N1CCN(c2ccc(F)cc2)CC1', 'CN(C)c1ncccc1C(=O)Nc1ccc(N2CC[NH+](CC(Cl)(Cl)Cl)CC2)cc1', 'Clc1ccccc1C[NH+](Cc1ccccc1Cl)[C@@H]1CCN(C(=O)OC(F)(F)F)C1', 'Clc1ccccc1OCC1[NH2+]CC1', 'Clc1ccsc1[C@@H](CO)CC[C@H]1CCCCO1', 'O=C(c1cnc(Nc2ccccc2)s1)N1CCC[C@H](c2cc[nH+]cc2)C1', 'Oc1cc(-n2nnc(Cc3ccc(Br)s3)n2)ccc1[O-]', 'Clc1cc(F)c(C2C[C@@H](CO)CC2)nc1S(=O)(=O)N1CCCCC1', 'NS(=O)(=O)N[C@@H](CCCn1cc[nH+]c1)c1ccc(F)cc1', 'O=C([O-])/C(=N/O)N1CCOCC1', 'C=C(C=S)C(=O)NCC1(O)CCCCC1', 'O=[N+]([O-])c1c(Cl)nsc1Cl', 'O=[N+]([O-])c1c(Cl)nsc1Cl', 'O=C([C@H](SC1NN(c2ccccc2)CC1)N1C[C@@H](CO)CC1)c1ccccc1', 'Clc1ccccc1[C@H]1[C@@H](C2CC2)Nc2ncnn21', 'CCO[P@](=S)(C(=O)[O-])c1ccccc1', 'O[C@@H]1C[C@H]1C(=O)Nc1ccc(Cl)s1', 'NS(=O)(=O)/C=C/C1CC1', 'COCc1c(C)sc2ncnc(N/C(=N/O)S(C)(=O)=O)c12', 'Nc1cn[nH]c1Br', 'OCc1n[nH]c(N/N=C/c2ncc(Br)cn2)n1', 'Oc1ccccc1[C@H](c1ccccc1)N1CCC(C[NH2+][C@@H](CO)C(F)(F)F)CC1', 'O=C([O-])/C(C#N)[C@@H](c1ccccc1Cl)c1c[nH]c2ccccc12', 'Clc1ccccc1CCC1CC[NH+](CCOc2ccc(C(=O)[O-])cc2F)CC1', 'O=C(c1cnccc1)[C@H]1C[C@@H](O)CO1', 'Oc1ccccc1-c1c([O-])cc([N+](=O)[O-])c2noc(=O)n12', 'Clc1ccc(Br)cc1/N=N\\C(=C\\c1nc2c(Cl)cccc2s1)C(=O)[O-]', 'Oc1ccc(F)cc1Cn1cnc(C[NH2+]C2CCOCC2)n1', 'Clc1cccc(C[C@@H](Cc2cccnc2)[NH+]2CCN(C(=O)c3ccccc3)CC2)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])ccc(NCCC[NH+]2CCCCC2)c1Cl', 'C=C1c2c(cccc2C)[C@H]1CC(=O)N[C@@H]1CCc2cc(SC)ccc21', 'Oc1cc(-n2cc[nH+]c2)ccc1/N=C/c1ccccc1[N+](=O)[O-]', 'Clc1cccc(F)c1N[C@H]1CC(Cc2ccccc2F)=[NH+]1', 'Clc1cccc([S@@]C2=NC(=O)[C@@H]3CCC(=O)N32)c1', 'C=C[C@@H](SC)c1nc(-c2cc(F)ccc2F)n[nH]1', 'Clc1ccc([C@@H](NN[C@@H]2CCc3ccccc32)[NH+]2CCCC2)cc1', 'N[C@@H](Sc1ccc(Cl)s1)C(=O)OCC1CC1', 'Clc1ccccc1Oc1cnc2ccc(C3(c4ccccc4)c(=O)n3[C@H]3CCOC3)nc2c1', 'Clc1ccc([C@@H](NSc2nncc3c2CCC3)C(F)(F)F)cc1', 'Clc1cccc(-c2ccc(C[NH+]3CCCC3)o2)c1', 'Clc1ccc(CC[NH+]2CCC(c3nccs3)CC2)cc1', 'Clc1ccc(Br)cc1C[NH+]1CCC(N2CCOCC2)CC1', 'O=C([O-])[C@H]1COc2cc(Cl)ccc21', 'Clc1ccccc1[C@H]1CCC[NH+]1CCCN1CCN(c2cccc[nH+]2)CC1', 'Clc1cccc2c(N[C@@H]3CCOC3Cl)ncnc12', 'OC[C@@]1([NH3+])CCc2ccccc21', 'COc1nn[nH]c1CNC(=O)NCc1ccc[nH+]c1N(C)C', 'Clc1ccc(-c2ccccc2[N-][C@@H]2CN[C@@H](c3ccc(Cl)cc3)C2)cn1', 'COc1ncncc1C[S@@](=O)Cc1nc(-c2cccs2)no1', 'O=C(CC(Nc1ccccc1)N1CCOCC1)N[C@@H](c1ccc([N+](=O)[O-])cc1)C(F)(F)F', 'O=C([C@H](Nc1ccccc1)[NH+]1CCSCC1)c1ccccc1', 'O[C@@H]1c2cc(Br)ccc2C[C@H]1Cl', 'C[C@@H](c1nnc(S[C@@H](C)C(=O)N2CCCc3ccccc32)s1)n1cccn1', 'O=[N+]([O-])c1c([O-])[nH]c2nccnc12', 'C=C[C@@H]1Cn2c(nnc2S[C@H](C)C(=O)N(CC)Cc2ccccc2)C1', 'OCS[C@H]1CC[C@](NN=C2/C(=CCc3ccccc3)NN2Cc2ccccc2)N1', 'Clc1ccccc1CCOc1ccccc1C[NH+]1CCC[C@@H](O)C1', 'O=C(/N=C\\c1ccco1)[C@@H]1CCCc2sccc21', 'Oc1cccc(Nc2[nH+]c3ccc(F)cc3s2)c1', 'O=C(c1ccsc1C[NH+]1CCCCC1)c1ccc(Br)cc1', 'O=C(/N=C/NC[C@](O)(c1cccc(Br)c1)c1ccccc1)c1cccc([N+](=O)[O-])n1', 'O=C(c1c[nH]c([O-])n1)N1CC(c2nc(-c3ccco3)nn2CC(F)F)c2ccccc21', 'Oc1cccc(CC2(O)C[C@@H]2c2ccco2)c1O', 'ClC([O-])=N/N=C/c1ccc(O)cc1O', 'Cc1sn(-c2ccc(F)cc2)c(=O)c1/C=C(/C#N)C(=O)[O-]', 'Oc1ccccc1-c1noc([C@H]2CCOC3(CCCC3)C2)n1', 'Clc1ccsc1/N=C/c1ccc(Sc2ncc[nH]2)o1', 'Clc1ccc(Br)cc1S[C@H]1C[C@@H](O)CN1C(=O)C1CC1', 'Clc1ccc(CN(C(=O)CCC2CC[NH2+]CC2)C(F)(F)F)cc1Cl', 'Clc1nncc(F)c1I', 'Clc1cccc([C@@H]2Nc3cc(OCc4ccco4)cc(Br)c3[C@@H]2C(=O)[O-])c1', 'O=C(/N=C/c1csc2ccccc12)N[C@@H](CC#N)c1cccc(Cl)c1', 'O=C(Cc1c(Cl)cccc1Cl)Nc1nnn[n-]1', 'NS(=O)(=O)c1cccc(NCC[NH+]2CCCCC2)c1', 'Oc1ccc([N+](=O)c2onc(C3CC3)n2)cc1OC[C@@H]1CCCO1', 'Oc1cc(F)ncc1S[C@H](C(=O)[O-])c1ccccc1', 'Oc1ccc(F)cc1-c1csc([C@@H]2Oc3ccccc3O2)[nH+]1', 'COc1nnn(CCCNC(=O)[C@H]2CCO[C@H]2C)c1[N+](=O)[O-]', 'O=[N+]([O-])c1c([N+](=O)[O-])cnc2nc(C[S@@](=O)Cc3ccncc3)[nH]c12', 'Clc1cccc(C[NH2+]Cc2nc(C3CC3)ns2)c1', 'Oc1ccccc1[C@H]1O[C@@H](CO)[C@@H](O)[C@H](O)[C@@H]1O', 'C=C(Br)CN(C(=O)c1cccn1CC)[C@@]1(C)CCCO1', 'Clc1ccccc1Cc1cn(C[C@@H]2CCCC[C@H]2[NH3+])nn1', 'Oc1ccc(/C=N\\C2=C(C(=O)c3nncn3CCc3ccncc3)[NH+](C)C2)cc1', 'Clc1ccccc1NC1C2CCC(C2)C1', 'Clc1cc(/N=C\\c2c([O-])[nH]c3ccccc32)[nH]c(=O)c1Br', 'O=C(c1c[nH]cn1)N1CC[C@@](CO)C1', 'Clc1cccc2c1N[C@@H](c1ccc([N+](=O)[O-])cc1)[C@@H]1CC=C[C@@H]21', 'O=C(c1c[nH]cn1)[C@H]1CCS(=O)(=O)C1', 'ClC([O-])(Br)Cc1cccc(Cl)c1Cl', 'O=C(C[NH+]1CCOCC1)Nc1cccc(Cl)c1', 'O=C(CNC(=O)CS[C@@H]1N=NC=C1O)NC[C@@H](CO)c1ccccc1', 'C=C(C(=O)CN1C(=O)COc2ccc(S(=O)(C)C)cc21)c1ccccc1', 'Clc1ccc(Br)cc1/C=N\\[C@H]1CCC[C@H](O)C1', 'Oc1cccc(-c2c[nH]nc2CN2CCNC(=O)[C@H]2c2cccs2)c1', 'CC[S@](=O)[C@@H]1CCCC[C@@H]1NC(=O)c1cccnc1', 'Oc1cc(Br)c2sncc2c1F', 'C[C@@H](Nc1ncccn1)[C@@H]1CCCN(c2ncccc2C(C#N)=O)C1', 'Clc1cc(N2CCOCC2)nc(C[NH+]2CCC[C@]3(CCC(=O)N(Cc4ccccc4)C3)C2)c1', 'O=C(COc1ccc2nccnc2c1)N1CC[C@H]([N+]2c3ccccc3CS2)C1', 'Clc1nnn(-c2nccs2)n1', 'Clc1ccc(CN2N=C3c4ccccc4O[C@@H]3C2)s1', 'ClC[C@@H]1CCO[C@]1c1c(F)cccc1F', 'OC[C@@H](Br)C1CCC(C(F)(F)F)CC1', 'Clc1ccccc1Cn1cc[nH+]cc1=S', 'O=C(C[NH+]1CCOCC1)NCCCc1ccc(F)cc1Cl', 'Clc1ccsc1/C([O-])=N/Oc1cncc(Cl)c1', 'Clc1ccccc1Oc1cnc([C@@H]2CCOC3(CCCCC3)C2)nc1N', 'O=[N+]([O-])c1c(-c2ccc(Br)cc2)n[nH]c1N[C@H]1CCC[C@@H](C(F)(F)F)C1', 'Oc1ccc(F)c(-c2[nH+]c3ccc(Cl)cc3o2)c1', 'Cc1snnc1CNC(=O)N(C[C@H]1COCCO1)C1CC1', 'N=CC1=C[C@@H](Sc2nnnn2C2CCC(C(N)=O)CC2)C(=O)N1c1ccccc1', 'Clc1nnn(-c2ncn([C@@H](C)CCN3CCOCC3)n2)c1', 'Cc1ccc(C(=O)CC[n+]2cccc[nH+]2)cc1F', 'O=[N+]([O-])c1ccncc1SC[C@@H]1C[C@H](O)c2ccccc2O1', 'O[C@@H]1c2ccoc2C(=O)N1[C@H]1CCC[C@@H]1CC(=O)Nc1ccc(F)cc1', 'Oc1cc(Br)cc(F)c1N1CCN(c2[nH+]cccc2[N+](=O)[O-])CC1', 'Oc1cc(O[C@@H]2CCN(c3cccc(Cl)c3)S2)cc2c1OCCO2', 'O=[N+]([O-])c1ccc2[nH]ncc2c1/N=N/c1csc(Br)c1F', 'Oc1ccc(/C=C(/CO)C[NH+](C)CCCS)cc1NC(=O)c1ccccc1', 'O=C(c1ccc(O)c(Cl)c1)N1CCC([NH+](Cc2ccco2)C2CC2)CC1', 'Clc1ccccc1-c1nncn1[C@@H]1CC[C@@]2(CCCN(Cc3ccc(F)cc3)C2)C1', 'Clc1ccc(CCC[NH2+]C[C@H]2CCC[C@@H]2c2ccccc2F)c(F)c1', 'O=C(CC[C@@H]1CCCO1)NCCc1cccc([N+]2=CCCC2)c1', 'Oc1cc(-n2cnc(Cl)n2)ccc1OCCN1CC[NH2+]CC1', 'O=C(/N=C\\NCSc1cccc(Cl)c1)N1CCOC[C@@H]1C1CC1', 'Oc1ccc(O[C@@H](C[NH+](C)C)Sc2ccccc2)c(Cl)c1', 'O=C(c1cn2ccsc2c1N1C[C@@H]2CCCC[C@H]2C1)Nc1cccc2cccnc12', 'Clc1ccccc1NS[C@H]1Oc2cc(Cl)ccc2S1', 'O[C@@H](Sc1nncn1Cc1c[nH]nc1c1ccccc1)C1CCOCC1', 'Clc1ccsc1SC[C@H]1Cc2ccccc2S1', 'C[C@@H](c1nc[nH]n1)n1ccc(NC(=O)c2ccc(C#N)cc2)n1', 'Oc1cc(O[C@@H]2O[C@H](CO)c3ccccc32)ccc1OCC(F)(F)F', 'O=C([O-])[C@@H]1C[C@H]1c1cccc(Cl)c1Cl', 'Clc1ccc(Br)c(F)c1Cn1c[nH+]c(-c2cccc(Cl)c2)n1', 'C=C1Nc2ccc(C#N)cc2[C@H]1c1cccc(Br)c1OC(F)F', 'C=C[C@@H](O)[C@H](C)c1cccs1', 'CNS(=O)(=O)C[C@H](C)[NH2+]Cc1ccc(Cl)c2ccc(F)cc12', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1[O-]', 'Clc1cc(/C=N/O)c(I)c(C(F)(F)F)c1[N+](=O)[O-]', 'Clc1cc(C2=NN=C([S-])N2)ccc1OCO', 'CCOC(=O)N1CCC2(CC1)C[NH+]=C(N)N2c1ccccc1', 'Cc1snnc1S[C@H](C)C(=O)N(C)Cc1ccc(=O)[nH]c1', 'Nc1[nH]nc2nc(CN3C[C@H](CCO)C[C@@H]3c3ccccn3)ccc12', 'O=C(c1c[nH]cn1)[C@H]1Oc2ccccc2NC1=O', 'Oc1cc(OC[C@@H](O)C[NH2+]C2CC2)ccc1OCc1cccnc1Cl', 'Clc1ccccc1C(=O)O[C@H]1C=NN=C[C@@H]1c1ccccc1', 'O=C(c1cc(Cl)ccn1)N1CC[NH2+]CC1', 'O=C([O-])COc1nc(CCc2ccsc2)n[nH]1', 'Oc1ccc(Oc2nc(N3C[C@H](C)O[C@H](Sc4ccc(F)cc4)C3)no2)cc1', 'Clc1ccc(COc2ncc(C[NH+]3CCCC3)[nH]2)cc1', 'NS(=O)(=O)/N=c1\\[nH]c(-c2cccs2)nc2cc(C(=O)[O-])ccc12', 'Oc1ccc([C@H](N[C@H]2CCC[C@@H]2CO)N2CCOCC2)cc1', 'O=C(/C=C/c1ccn[nH]1)O[C@@H]1CCC[C@H](C(F)(F)F)C1', 'Oc1cccc(C(F)(Cl)NNC(=O)c2csc(Br)c2)c1', 'Oc1cc(-n2cnc(Sc3nc(C4CC4)ns3)c2[O-])ccc1O', 'CNc1ccc(Cl)c(C)c1C(=O)N(C)[C@@]1(C)CCCC[C@@H]1C', 'Clc1ccccc1O/C=C[C@@H]1CN2CCCC[C@H]2CO1', 'Clc1cccc2nn(SC[C@@H]3CCCO3)nc12', 'Clc1cc(/C=N/n2cccc2)nc(C(=O)[O-])c1', 'N[C@@H]1c2ccccc2[C@@H]2OCC[C@H]21', 'Oc1cccc(S[C@@H]2CC(c3ccccc3)C(=O)N2c2ccc(C(C)=O)cc2)c1C(F)(F)F', 'Oc1cccc2nc(CN3CCCC[C@H]3C(=O)[O-])nn12', 'OC1=C2CCCC[C@@H]2C(=O)N1C(=O)c1ccc(Br)s1', 'ClC(c1cccs1)c1cc(N2CCN(C(=O)Cc3ccccc3)O2)ccn1', 'O=C(c1c[nH]nc1N1CCOCC1)N1CCN(C[C@H]2CCCC[NH+]2Cn2cncn2)CC1', 'Nc1c(/N=N/NC(=O)c2ccnc(OC3CCC3)c2)[nH]c2c1CCC[C@H]2O', 'Clc1nnnc(Cl)c1I', 'O=C([O-])c1nnsc1Sc1ccccn1', 'O=[N+]([O-])c1c(O[C@@H]2CCCC[C@@H]2C(=O)[O-])ccc2cccnc12', 'Clc1nnn(-c2ncn[nH]c2=O)n1', 'Nc1cn[nH]c1[C@@H]1N=C(C2CC2)c2nc3ccc(Cl)cc3n21', 'ClC(=O)N[C@H](Cc1cccs1)C(=O)[O-]', 'Clc1ccc([C@@H]([NH2+]C2CCN(C[C@@H]3CCOC3)CC2)C2CC2)cc1', 'Clc1ccccc1O/Cn1cnn[nH+]1', 'Clc1cc(Br)ccc1O[C@@H]1CCOC2(CCC2)C1', 'Clc1ccc([C@@H](NS[C@H]2CCOC3(CCCC3)C2)N2CCOCC2)cc1', 'OC[C@H]([S@@](=O)C1CCOCC1)N1CCCc2ccccc21', 'Clc1nnc(N2CC=C(n3cc(-c4ccccc4)s3)CC2)o1', 'NS(=O)(=O)NC1CCC2(CC1)OCCO2', 'Clc1ccccc1Oc1cnc(N[C@@H]2CCC[C@@H](C(F)(F)F)C2)[nH]1', 'OCc1nnc2n1CC[C@H]2O', 'Cc1sncc1[C@@H](C)Oc1cnc(Oc2cccc(Cl)c2)c(C#N)c1', 'NS(=O)(=O)C[C@@H]([NH3+])c1cccc(Cl)c1', 'NS(=O)(=O)C[C@@H](CNc1nc2ccc(Cl)cc2s1)N[C@@H]1CCS(=O)(=O)C1', 'O=[N+]([O-])c1c(F)ccc(F)c1C(=O)NCCC[NH+]1CCCCC1', 'ClCc1csc2nc(N3CCC[C@@H](C(=O)N4CCC[C@H]4C)C3)nn12', 'O=C(CC[NH+]1CCC[C@H](c2nc(C3CC3)no2)C1)Nc1ccc(Cl)cc1', 'Clc1cc(Br)cn1N[C@@H]1C[C@H](O)CN1C(=O)Nc1cccc2ccccc12', 'Clc1ccc([C@@H](CCN2CCOCC2)[C@@H]2CCS(=O)(=O)C2)s1', 'Clc1cc(Cl)c2ccccc2c1C[NH+](Cc1ccccn1)C1CCCCC1', 'O=C(/N=C/CC1CC[NH+](Cc2ccccc2)CC1)C1CCCC1', 'Clc1cc(F)cc2nnc(N[C@@H]3CCCOC3)n12', 'ClC(/N/c1c(Cl)cccc1Cl)(SCc1c[nH]c(=O)[nH]1)Cc1ccccc1', 'O=C(Cc1c[nH]nc1C(=O)NC[C@H]1CN2CCCC[C@@H]2CO1)Nc1nccs1', 'Clc1ccccc1C[NH+]1CCc2ccccc2[C@H]1C(=O)Nc1cccc(Cl)c1', 'N[C@@H]1[C@H](OCc2ncc[nH]2)OC[C@@H](OCc2ccccc2)C1', 'COC[C@H](O)CN(C)C(=O)[C@@H]1CCC(=O)NC1', 'COCc1nc(C(=O)N2N=CN(CC#N)CC2)cs1', 'O=[N+]([O-])c1ccc(C[NH2+]Cc2ccc(O[C@@H]3CCOCC3)o2)cc1', 'Clc1ccccc1-c1ncc(F)cc1C[S@@](=O)c1cscc1C(F)(F)F', 'Nc1cn[nH]c1', 'Nc1cnn(CCC(=O)N2CC[C@@H](C(=O)[O-])[C@H]2C)c1[N+](=O)[O-]', 'Clc1ccsc1C[N-]Sc1ccc(Br)cc1', 'C[C@@H]([NH2+]C[C@@H]1CCCC[C@H]1CCOc1ccccc1)c1ccc(F)c(F)c1', 'Clc1nnn(-c2cncc(C(=O)[O-])c2)c1Cl', 'O=C([C@H](C#N)c1cccs1)N1CCC[C@@H](c2ncccn2)C1', 'ClC(=S)[N+]1(c2cc(Br)ccc2F)CC1', 'O=C(C1=N[C@@H](CC(F)F)N1c1ccc(Cl)cc1)N[C@@H]1CCS(=O)(=O)C1', 'Clc1ccccc1CS(=O)(=O)C1CC[NH+](C2CCCCC2)CC1', 'Oc1cc(C=O)on1', 'O[C@@H]1c2ccccc2-c2nccnc2S1(=O)=O', 'Clc1nnn(C[C@@H](O)CO)c1Br', 'Clc1ccc(Br)cc1C1=c2ccccc2=[NH+]C1', 'Oc1ccc([N+](=O)[O-])c(C2=NC=CCS2)c1[N+](=O)[O-]', 'O=C(CNc1ncon1)N[C@H]([C@@H]1CCS(=O)(=O)C1)c1ccccc1', 'Oc1cccc(SC2CCC([NH2+]Cc3cccs3)CC2)c1', 'CNC(=O)[C@@H]1CCCCN1c1ccc(NC(=O)[C@H](C(C)C)[NH+]2CCCCC2)cc1', 'O=[N+]([O-])c1cccc(-c2nnc(CC[NH+]3CCCC3)o2)c1', 'O=C(/C(CNN=C1CCCC1)c1ccc(Cl)cc1)CC1CC[NH+](C2CCCCC2)CC1', 'Oc1ccc(/C=C/N=[N+]2CCC[C@H]2SC2CCCCC2)cc1', 'Clc1cccc(S[C@H]2C[NH2+]CCN2C(=O)c2ccccc2)c1', 'O=[N+]([O-])c1c(O)nc2ncnc(NC3CCC3)c2c1C[NH+]1CCCCC1', 'O=[N+]([O-])c1c([O-])[nH]c(C[NH+]2CCC(n3cccn3)CC2)c(-c2ccccc2)c1=O', 'C[C@@H](C[NH2+][C@@H]1CCN(c2ccccc2F)C1)N1CCOCC1', 'OC[C@@]1(N[C@@H]2CCS(=O)(=O)C2)CCCC1', 'OCS1(=O)CCN(C(=O)c2ccc(F)cc2F)CC1', 'Oc1ccccc1OS[C@H](c1ccccc1)N1CC[C@@H]([N+](=O)[O-])C1', 'Clc1cccc(F)c1[C@@H]([O-])[C@H](CSc1ccc(F)cc1)N1CCCC1=O', 'Nc1nonc1C=O', 'Clc1ccccc1-c1cnnc(O[C@@H]2CCOC3(CCCC3)C2)n1', 'CCc1[nH]c(CCl)c(C#N)c1N', 'NS(=O)(=O)C[C@H](SCc1ncoc1C1CC1)c1ccccc1', 'O[C@@H]([C@@H]1CC(C(C)C)=C(C)CS1)c1ccc(Cl)cc1', 'Clc1nnnn1-c1c(Br)ccc(Br)c1[O-]', 'O=C([C@H]1[C@H]2C=C[C@@H](CN2c2ccccc2)C1)N1CCC(Cn2ccnc2)CC1', 'Clc1cc(Cl)nn1/N=C(/[O-])c1ccc2c(c1)OCO2', 'Clc1ccc([C@@H]([O-])C[C@@H](/C=O)c2cc(F)ccc2F)cc1', 'O[C@@H]1c2ccc3ccccc3c2CN1C(=O)CS[C@H]1CCS(=O)(=O)C1', 'NS(=O)(=O)/C=c1/sc2nc[nH]c2cc1NC(=O)c1ccccc1F', 'Nc1[nH]nc2c1/C(=C(/[O-])Nc1ccc([N+](=O)[O-])cc1)S2', 'Clc1cc(F)c(O[C@@H]2CCC[C@@H](C[NH3+])C2)cc1', 'N#Cc1nsnc1CCC(=O)NC1CCc2cccc(F)c21', 'Clc1nnnc(O)c1Br', 'Clc1ccc([C@@H](Br)C[C@@H]2CCOC3(CCCC3)C2)cc1', 'Clc1ccc(Cc2nnn[n-]2)cc1', 'O=C(C1CC[NH+]([C@H](c2ccccc2)[C@H]2CCCO2)CC1)N1CCOCC1', 'O=C([C@H](SC[C@@H]1CCCCO1)c1c[nH]c2ccccc12)C(=O)[O-]', 'ClCc1ccn([C@H](OC(F)F)C2CCCCC2)c1', 'COCc1noc([C@@H]2CC(=O)c3[nH]nc(C)c3C2)n1', 'CN=N/NC(=O)[C@H](C)S(=O)(=O)C[C@@H](C)c1ccccc1', 'CC[S@](=O)[C@H]1CCCC[C@H]1NC(=O)C(=O)Nc1ccc2[nH]cnc2c1', 'Oc1cc(F)ncc1-c1ncc(Cl)cc1O[C@H](CO)C(=O)[O-]', 'O=C([O-])CS/N=C1\\C(=O)Nc2c1cnc1ccccc21', 'Clc1nnnn1CC[C@@H]1CSc2ccccc21', 'N[C@@H](CCc1nccs1)[C@H](c1ccccc1)[n+]1ccccc1', 'Clc1ccc(C[NH+](CC2CC2)C2CC2)cc1', 'Clc1cc(Cl)nc([O-])c1F', 'OCSc1nnc(N2CCC[C@@H]2[C@@H]2CCCC2)[nH]1', 'Clc1cccc(CCn2c(-c3cc(Cl)ncc3[N-]S(=O)(=O)c3ccccc3)no2)c1', 'Clc1cccc2ccc(/C=C3\\SC(=S)N(c4nonn4)C3)cc12', 'Clc1ccc([C@@H](Cl)[C@H]2CN3CCC[C@@H]3CO2)cc1', 'O[C@@H]([C@@H](c1ccc(F)cc1)[C@@H]1CCOC1)N1CCOCC1', 'Oc1ccccc1O[C@@H]([C@@H]1CCCN(c2ncccn2)C(F)(F)F)C1', 'ClC[C@H](Br)[C@@H]1CCCN(S(=O)(=O)c2cccs2)C1', 'Clc1cccc2ncnc(NCC[NH+]3CCC(O)CC3)c12', 'Clc1cc(NC[C@@]2(O)CCSC2)ccn1', 'O[C@@H](S[C@@H](F)c1cccnc1)c1ccccc1F', 'COc1nnn(C)c1C[C@H](C)OC(=O)c1cc(C)n(C)n1', 'N[C@@H](c1cccs1)CO[C@@H]1C[C@H]1C', 'NS(=O)(=O)CC[C@@H]1N=Nc2ccccc21', 'Oc1cc(F)ncc1C[NH2+]C[C@@H]1CN(Cc2ccccc2F)CCO1', 'O=[N+]([O-])c1cc(C2/N=CC=C[C@@H]2Oc2ccccc2)ccc1Br', 'O=[N+]([O-])c1c([N+](=O)[O-])c(F)cc(Br)c1[O-]', 'OC[C@@]1(c2ccccc2)Nc2sccc2N1', 'Clc1cccc([C@@H]2[C@@H]3CCC[C@@]3(CO)C2)c1', 'O=C(c1ccsc1C#N)N1CCN([C@@H]2C=CC[C@@H]2C(=O)c2ccccc2)CC1', 'CN=N[C@]1(C(=O)OC)CCc2ccccc21', 'Nc1[nH]ncc1C[NH+]1CCC(Oc2ccccc2F)CC1', 'Clc1ccc([C@@H]([C@@H]O)N[C@@H]2CCC[C@@H](O)C2)cc1', 'Oc1ccc(F)c(Cl)c1N[C@@H]1CCS/C1=N/c1cccc(Br)c1', 'O=C(c1c[nH]cn1)O[C@@H]1CCS(=O)(=O)C1', 'Clc1ccc(-c2cccnc2)nc1O[C@@H]1CCOC2(CCCCC2)C1', 'Clc1ccccc1NS[C@H]1CCCC[C@@H]1C(=O)[O-]', 'O=[N+]([O-])c1ccc(S[C@@H]2CCCS[C@@H]2c2cccs2)cc1', 'NS(=O)(=O)Cc1noc(I)c1[N+](=O)[O-]', 'Clc1nnc(S[C@@H]2CCC[C@@H](O/N=C/c3ccccc3)C2)n1C1CC1', 'O=[N+]([O-])c1c(Br)c([C@@H]2CCCCO2)cs1', 'N[C@@H](Sc1nnc2ccccn12)[C@@H]1CCCO1', 'Clc1ccccc1NC1(c2ncc[nH+]c2Cl)CCOCC1', 'Clc1ccc([C@@H](OCC(F)F)c2cccnc2)C[C@@H]1CC(=O)Nc1ccc(F)cc1', 'NS(=O)(=O)Cc1noc(Cn2c(=O)ccc3nc(C(=O)[O-])cn32)n1', 'Clc1nnnn1/C(\\[O-])c1ccccc1C(F)(F)F', 'Clc1cc(Cl)ncc1[O-]', 'Clc1cc(/C=C2/S[C@H]3CS(=O)(=O)C[C@@H]3N2c2ccccc2)ccc1[N+](=O)[O-]', 'Oc1ccc(Oc2ccccc2C[NH2+]C2CC[NH2+]CC2)cc1', 'Nc1cn[nH]c(=S)n1', 'Clc1ccc([C@@H](Br)[C@@]2(O)CCSC2)cc1', 'OC[C@H]1CCCC[NH+]1Cc1cccc(OCc2ccccc2)c1', 'COc1nn(-c2ccccc2)c(C)c1C(=O)NC[C@@H](c1cccs1)[NH+](C)C', 'O=[N+]([O-])c1ccc[nH+]c1NCCSc1ncccn1', 'Clc1ccc(-c2cccs2)c(NC[C@H]2[C@@H]3CC[C@@H]2C3)c1', 'O=C(/N=C\\N=c1\\scc(-c2ccc([N+](=O)[O-])cc2)[nH]1)c1ccc[nH]1', 'O=C([C@H](N[C@@H](C(=O)[O-])C1CCCC1)Nc1cccc(Cl)c1)OCCO', 'NS(=O)(=O)N(C)CC(=O)N[C@H]1CCCC[C@@H]1OC1CCCCC1', 'Clc1cc(F)c(N[C@H]2CC[NH2+]C[C@H]2Cc2ccccc2)cc1', 'N#C[C@H]1C(=O)NC[C@@H](C(=O)c2ccccc2)N1CNc1cccc(Cl)c1', 'Clc1ccsc1SC/C=C/c1nc2sccn2c1Cl', 'ClCc1ccn(Cc2nc(N3CCOCC3)nc[nH+]2)c1', 'OC[C@@H](C1CCSCC1)N1CCC[C@@H](c2cc(-c3ccccc3)no2)C1', 'Clc1nnnn1/N=C/Sc1ccc(C(F)(F)F)cc1Cl', 'Clc1ccccc1NS[C@H]1CCOC2(CCCC2)C1', 'O=C([C@H](Nc1nc[nH]n1)C1CCCC1)N(C1CCCCC1)[C@@H]1CCS(=O)(=O)C1', 'Oc1cc(COc2n[nH]c(=O)[nH]2)ccc1C[NH2+]Cc1ccco1', 'Clc1nncc(Cl)c1N[C@H]1COCCN1CC1CC1', 'Clc1ccc(CCBr)c(CN2CC[NH2+]CC2)c1', 'OCc1n[nH]c(-c2ccc(N3CCCC3)c[nH+]2)c1[N+](=O)[O-]', 'O=C([O-])/C=C/[C@H]1Cc2ccccc2O1', 'Clc1cccc(C[NH2+][C@@H]2CCCC[C@@H]2c2cccc(F)c2)c1', 'Clc1cccc(/C=C/C=C/CC[NH2+]C2CC2)c1', 'O=C(/C(Cc1ccco1)c1ccc(F)c(Cl)c1)Cn1c[nH+]ccc1=O', 'O=[N+]([O-])c1c(Cl)nc(Br)s1', 'Nc1nnn([C@H](CCO)C2CC2)n1', 'Clc1ccccc1C[NH+](CCCN1CCOCC1)C1CC1', 'Clc1cc(CO[C@H]2CSCCS2)c(O)cc1O', 'OCc1nnsc1Br', 'Clc1nnnc(CI)c1', 'O=C([C@H](Sc1cc(Cl)ccc1Cl)n1cccn1)Nc1n[nH]c(=O)[nH]c1=O', 'O=C(c1cncc(I)c1)[C@@H]1CN2CCC[C@@H]2CO1', 'Clc1ccc(-c2nonc2C[NH+]2CCCC2)cc1', 'Clc1ccc(Br)cc1NC[C@H](c1ccccc1)[NH+]1CCCC1', 'O=C([C@H](C(F)(F)F)[NH+]1CCN(CCO)CC1)Nc1ccccc1C(F)(F)F', 'Clc1ccc(C[NH2+]c2cccc(C(=O)[O-])c2)cc1', 'O=C([C@H](NS(=O)(=O)c1cccc2ccccc12)[NH+]1CCCC1)c1ccc(F)cc1Cl', 'ClCc1csc([N-]S(=O)(=O)c2cccc(F)c2)n1', 'O=C([C@H](NSc1ccccc1)[C@H]1CSC(=O)O1)c1ccccc1', 'O=[N+]([O-])c1ccc[nH+]c1N1CCC(n2cncn2)CC1', 'O=C(C1=c2ccccc2=[NH+][C@@H]1C(F)(F)F)N1CCN(Cc2ccc([N+](=O)[O-])cc2)CC1', 'Clc1nnn(C[C@@H](CO)C2CC2)c1Cl', 'Oc1cc(-n2cc[nH+]c2C2CC2)cc(N2CC(=O)Nc3ccc([N+](=O)[O-])cc32)c1', 'Clc1cc(Cl)nnc1S[C@@H](C[NH3+])c1ccccc1', 'Clc1cc(Br)c(C[NH3+])nc1NC[C@@H]1CCCO1', 'Clc1cc(NSc2nncc3c(O)cccc23)no1', 'O=C(/C=C1/S/C(=C/c2cccc(F)c2)N1c1cccnc1)N1CCCC[C@@H]1CNc1ccc2c(c1)CCCO2', 'Oc1ccc(F)c(C[NH2+][C@H]2CCCc3ccccc32)c1', 'O=C(/C(Cc1ccccc1)[NH+]1CCCC1)c1ccc([N+](=O)[O-])cc1', 'ClC[C@H](C[C@@H]1CCCO1)c1ccc(Br)s1', 'O=C(Cc1cnn(Cc2ccccc2)c1)NC[C@@H](CCO)[NH+](C)[C@@H](C)c1ccccc1', 'Clc1ccc(-c2cccs2)nc1NCC[NH+]1CCCCC1', 'Oc1cc(CC[NH2+][C@H](c2ccccn2)c2ccco2)ccc1F', 'Clc1cccc(/N=N/[C@@H]2C=CCC2)c1', 'Nc1[nH]ncc1S(=O)(=O)N1C[C@H]2CC=CC[C@@H]2C1', 'Clc1cccc(Br)c1N1CC[NH+](Cc2cccnc2Cl)CC1', 'Clc1ccc(Br)cc1[C@@H]1COC2(CCCC2)C1', 'Oc1cc(C[NH2+][C@H](c2ccccc2)c2ccc(F)cc2)c[nH]1', 'O=[N+]([O-])c1c([O-])c2sccn2c1C1CC1', 'Oc1ccc(/C=N\\CC[NH+]2CCCCC2)cc1', 'Clc1ccc(C[NH2+][C@H]2CCC[C@@H]2[C@@H]2CCCC2)cc1', 'Oc1ccccc1/N=N\\Nc1cc([N+](=O)[O-])ccc1C[NH2+]C1CC1', 'Clc1ccc(CCCC[NH2+][C@@H]2CCC[C@H](C(F)(F)F)C2)cc1', 'Clc1ccsc1S(=O)(=O)[C@@H]1CCCO1', 'Clc1ccccc1[C@H]([NH3+])c1csc(Br)c1', 'CC[C@@H]1C(=O)NC[C@@H](C(=O)OC)C1', 'Clc1ccc(Cc2cc[nH]n2)c(C[NH2+]C2CC2)c1', 'Clc1nnnc(Br)c1Br', 'CCSC1=NN(c2ncccc2[N+](=O)[O-])C(=O)[C@@H]1C', 'Clc1cc(Cl)c2c(n1)CCC[C@@H]2[NH2+][C@@H](c1ccccc1)c1cccc(N2CCOCC2)c1', 'N[C@@H](C[NH+](C)CC(=O)N1CCOCC1)c1ccc(F)cc1', 'O=C([C@H](c1c([N+](=O)[O-])nnc2sccc12)[NH+]1CCCC1)N1CCN(c2ncccn2)CC1', 'Oc1cccc(-c2c[nH]nc2[C@H]2CC(c3ccccc3)=NS2(=O)=O)c1', 'Clc1ccc([C@@H]([C@@H]2Cc3cnccc3S2)N2CCOCC2)cc1', 'OC[C@@]1([C@H]([NH3+])Cc2ccccc2)CCOC1', 'Clc1ccccc1Oc1cnc(NCc2cccs2)nc1CC[NH+]1CCC[C@@H](Cl)C1', 'Clc1ccc(C[NH2+][C@@H]2CCO[C@]2(c2ccccc2)c2ccco2)cc1', 'Oc1ccc(O[C@@H](c2ccco2)[C@H](CO)C2CC2)cn1', 'N=Cc1cccn1CC(=O)N1C[C@H](C)OCC1(C)C', 'Clc1ccccc1-c1nnc(C[NH+]2CCN(Sc3ccccc3)CC2)c(=O)[nH]1', 'Oc1cc(O[C@@H]2ON=C(c3ccccc3)C[C@H]2C(=O)[O-])ccc1O', 'O[C@@H](c1cc([N+](=O)[O-])ccc1F)[C@H]1C[C@@H]1c1cc(Cl)ccc1Br', 'ClCc1ncoc1Cl', 'COc1nn(-c2c(C)cc(C)c(C)c2NC(=O)C(=O)N[C@@H]2CCCC[C@@H]2C)cn1', 'Oc1cc(C=NSCc2ccsc2)nc2sccc12', 'O=C(/C(CO)[C@@H]1CCC[C@@H](C(F)(F)F)C1)c1ccc([N+](=O)[O-])cc1', 'Clc1ccc(Br)c(OCOC2CC[NH2+]CC2)c1', 'OCc1n[nH]cc1OCC1CC[NH+](Cc2ccccc2)CC1', 'O=C(/C=C1CS(=O)(=O)C[C@H]1O)c1ccccc1Br', 'Clc1ccc(Br)cc1/N=N\\[C@@H]1CCCC[C@@H]1S(C)(=O)=O', 'CC1(C)[C@H]([NH2+]CNc2ccccc2)C(C)C1', 'ClC[C@@H]1CN2CCCC[C@@H]2CO1', 'NS(=O)(=O)C[C@@H](O)CN1CCN(C[C@H](O)C(F)(F)F)CC1', 'O=C([O-])C[C@@H](N[C@@H]1CCc2c(Br)cccc21)C(=O)[O-]', 'Clc1ccccc1[C@H]1NN[C@H](c2ccco2)N1O', 'O[C@@H]1C[C@@](O)(/N=C/c2cccn2Cc2ccc(Cl)cc2Cl)CC1', 'Clc1cccc2cc(C[NH+]3CCCC3)ccc12', 'O=C(c1cnc(NSc2ncccn2)nc1Cl)[C@H]1CCCO1', 'Cc1snnc1SCC(=O)Nc1cccc([N-]S(C)(=O)=O)c1', 'Clc1ccccc1Nc1cn([C@H]2CCOC3(CCOCC3)C2)[nH]c1=O', 'Clc1cc(/N=C/C(=N/O)c2sccc2C(=O)[O-])ccc1O', 'Clc1cc(/C=C(/NC2=NN=CC2)Sc2ccccc2)[n+]([O-])c2ccccc12', 'O=C(/C(c1cccs1)[NH+]1CCCCC1)c1cn(Cc2ccccc2)nn1', 'O=C(CSC(NCC1(C(=O)O)CCCC1)N1CCCC1)N1CCCC1', 'O=C([O-])[C@H]([C@@H](C(=O)n1cncn1)c1ccccc1)c1cccc(Cl)c1', 'CC(C)CC[C@H](C)[NH2+]Cc1cccc(O[C@@H](C)C(=O)OC(C)(C)C)c1', 'Clc1ccccc1[C@H]([NH2+]/N=C/c1cccc(Cl)c1)n1cnnn1', 'Nc1cn[nH]c1I', 'O=C(/C(c1cccs1)[C@@H]1CC=CC=C1O)c1ccccc1O', 'O=[N+]([O-])c1c(O)sc2c1CCN(Cc1cc[nH]n1)C2', 'O=C(c1ccco1)N1CC(Cn2cc[nH+]c2)c2ccccc21', 'O=C(c1cc([S@](=O)c2cccs2)ccc1N1CC[NH+](CC(=O)N2CCOCC2)CC1)[C@H]1CCOC1', 'O=C(/N=c1/[nH]ccn1CC1CC1)c1csc(-c2c[nH]cn2)n1', 'OC1CCC([NH2+]Cc2ccco2)CC1', 'O=C(/C=C\\c1nc2sccn2c1Cl)N[C@@H]1CCOC1=O', 'N=Cc1c(NC(=O)[C@]2(c3ccc(F)cc3F)NC(=O)C(=O)N2C)cccc1OC(F)F', 'ClC[C@@H]1Oc2cc(F)ccc2C[C@@H]1F', 'Clc1cccc([C@@H]2O[C@@H](O)CO[C@@H]2c2cccc(OC(F)F)c2)c1', 'ClCc1cn[nH]c1O[C@@H]1CCCC[C@@H]1OC1CCCC1', 'Clc1ccc(CC[C@H]([NH3+])[C@H]2COc3ccccc3C2)o1', 'CCNS(=O)(=O)C[C@@H](C)C#N', 'Clc1ccc(Br)c([C@@H]([NH2+]Cc2cnc3cc[nH]c3c2)C2CC2)c1', 'O=C([C@H](Cl)Br)S(=O)c1ccc2ccccc2c1', 'C=C1[C@H](C)C[C@H](C(=O)O[C@@H](C)c2ccccc2)[C@H]1O', 'Clc1ccc(-c2cc(/C([O-])=NO[C@@H]3CCC[C@@H](CO)[C@H]3NC(F)(F)F)nn2C2CCCCC2)cc1', 'C=C[C@@H](C(=O)OC(C)(C)C)N1C(=O)C(=O)N(C)C1=O', 'O=C(C1=CC[NH+](CCCN2CCOCC2)CC1)Nc1cccnc1', 'Clc1ccsc1[C@H]([O-])[C@@](C)(CO)c1ccccc1', 'ClCc1nnsc1[N+](=O)[O-]', 'Clc1ccc(C2=C[C@@H](O)[C@]3(CCCN(c4ccccc4)C3)C2)cc1', 'Clc1ccc(-c2cccs2)c2c1C[C@@H](Cl)[C@H]2Br', 'Nc1nncn1CCN1C[C@@H](C)OCC1(C)C', 'OC1(S(=O)(=O)c2ccc3c(c2)C=CS[C@@H]3c2ccccc2)CCOCC1', 'Oc1ccc(F)c(O[C@@H]2CCOC3(CCCC3)C2)c1', 'O=C(CNS(=S)=N/O)c1ccccc1', 'C=C1NC(=O)N([C@@H](C)c2ccccc2C)C(=O)[C@H]1/C=C/c1ccc2c(c1)OCO2', 'Clc1ccccc1C1(O)CC[NH+](C[C@@H]2C[C@@H]2c2ccccn2)CC1', 'CCn1cnc(N[C@](C)(CO)N2CCOCC2)c1C', 'N=Cc1nonc1[C@H]1NC(=S)N=C1N1CCN(c2ccccc2)CC1', 'C[C@@H]([NH2+]CC(=O)NC1(Cc2cccs2)CCC1)c1cccc(Cl)c1', 'Clc1ccc(Br)cc1C1CC[NH+](Cc2ccc(CO)o2)CC1', 'ClC[C@H]1O[C@]2(CCSc3ccccc32)O1', 'O=[N+]([O-])c1c(-c2n[nH]c([O-])n2)ccc2nsnc12', 'N#C[C@@H]([C@@H]1CCCON1c1ccccc1)c1ccc(-n2nnnc2CS)cc1', 'O=C(CCSCc1[nH+]ccn1CC(=O)[O-])c1ccccc1', 'Clc1ccccc1N1N=C(c2cccnc2Cl)C[NH+]1Cc1ccc(F)cc1', 'Clc1ccc(Br)cc1OC1CC[NH2+]CC1', 'Clc1ccc(COCO[C@H]2CCC[NH2+]C2)cc1', 'Nc1cncc(/[NH+]=C/c2ccc(C#N)cc2)n1', 'N[C@@H](NCC1(C)CC1)C(=O)N1CCC(C(F)(F)F)CC1', 'Cc1sc(C#N)c(S[C@@H](C(=O)[O-])c2ccc(Cl)cc2)n1', 'Oc1cc(F)ncc1N[C@H]1CCCn2ccnc21', 'COc1nc(-c2ccccc2F)cc(S(N)=[NH+]/CCNC(=O)c2ccc3c(c2)OCO3)c1Cl', 'COC[C@H]1[NH2+][C@@H](c2ccccc2OCc2cccc(C)c2)CC1', 'ClC(F)(CCO)N1CCc2sccc2C1', 'Clc1ccc(-c2csc3[nH]c(N[C@H]4CCO[C@@H](CO)C4)nc23)cc1', 'Oc1ccc([C@H]2C[C@@H]2C(=O)[O-])cc1F', 'C=C[C@@H](NS(=O)(=O)[C@H]1CCCN(c2ncc[nH]2)C1)c1ccccc1', 'Oc1ccc([N+](=O)C2=C([O-])O[C@@H]2c2ccccc2)cc1OCC(F)(F)F', 'NS(=O)(=O)Nc1cc([S@@](=O)Cn2cccn2)ccc1C(F)(F)F', 'Cc1c([N+](=O)[O-])cccc1N1C(=O)[C@H]2SCC[C@@H]12', 'Clc1ccc(Br)c([C@H]2OCC[C@@H]2CO)c1O', 'Clc1ccc(Br)c([C@H]2CCC[NH+]2Cc2ccccc2)c1', 'O=C([O-])C(C1CCOCC1)C1CCCC1', 'O=C(c1cn(C[C@@H]2CCCO2)cc1O)[NH+]1CCC(CN2CCOCC2)CC1', 'Clc1cccc(Br)c1C(F)(F)C[NH+]1CCCC1', 'Nc1c(F)cccc1N1CC[NH2+]CC1', 'ClC[C@@H]1C=C[C@@H](c2ccccn2)C1', 'O=[N+]([O-])c1cc(N[C@H](c2cccc(F)c2)C2CCCC2)c[nH+]c1', 'O=C([C@H]1CC[C@H](C[NH3+])O1)c1ccc(Br)cc1', 'OCc1n[nH]cc1O[C@@H](c1ccccc1)[NH2+]CCN1CCCC1', 'OC[C@@H](N[C@H]1CCSc2ccccc21)[C@@H]1CCC(=O)N1', 'N[C@@H](S/N=C(\\C)c1ccco1)N1CC=C(Cc2ccccc2)C1=O', 'Clc1cccc(/C(CC(=O)Nc2cc3c(cc2Cl)OCCO3)[NH+]2CCCC2)c1', 'O=[N+]([O-])c1c(F)cc(C[S@@](=O)/C=C(\\Cl)Cc2ccccc2)c(Cl)c1O', 'Oc1ccc(F)cc1CN[C@@H](C1CC[NH2+]CC1)c1ccc(F)cc1', 'Clc1ccc(CN[C@@H]2CCCC[C@H]2[S@@](=O)Cn2cncn2)cc1', 'Clc1ccc(CNC2CC[NH+](C[C@H]3CCCO3)CC2)cc1', 'ClC[C@H](CN1CCN(c2nc3ccccc3nc2C(F)(F)F)CC1)N1CC[NH+](CCCO)CC1', 'Clc1ccsc1/CC1(C#N)CCN([C@@H]2[C@H](CN3CCN(c4ccccc4)CC3)C2)C1', 'ClC[C@@H]1CO[C@H](c2ccccc2F)N1', 'Clc1ccc(-c2cccnc2)c([C@H]2N=c3ccccc3=[NH+]2)c1', 'NS(=O)(=O)/C(CNC(=O)CCc1ccccc1)[C@H]1CCCC[C@@H]1C(=O)[O-]', 'O=[N+]([O-])c1c(Br)sc2[nH]cnc12', 'C[C@@H]([C@@H](O)c1ccc2c(c1)CCCC2)[NH+](C)Cc1ccccc1O', 'Clc1cc(C[NH2+][C@H]2CCc3c(Br)cccc32)ccc1Cl', 'O=C([C@H]1CC[NH+](CCC2CC2)C1)N1CCN(c2ccc(Cl)cc2)CC1', 'Clc1cccc2cc(O[C@H](CSC(F)(F)F)C(F)F)ncc12', 'Clc1cccc(/N=C2\\O[C@H]3CCCC[C@@H]32)c1F', 'Clc1ccc(-c2nonc2[S-])nn1', 'Clc1nncn1C[C@@H](OCc1ccccc1)C1[C@@H](O)CO1', 'Oc1cc(Br)ccc1Nc1cnccc1C[NH+]1CCSCC1', 'O=[N+]([O-])c1cc(N[C@@H]2CCN(c3ccc(F)cc3F)N2)cs1', 'OC[C@@]1([NH3+])CCC[C@H](Cc2ccncc2)C1', 'Clc1nncc(S[C@@H]2CC[C@@H](COc3cc([N+](=O)[O-])ccc3F)C2)n1', 'ClCc1csc([N-]Sc2ccccc2)n1', 'O=C(c1cc([N+](=O)[O-])ccc1-n1cccn1)N1CC1(C#N)C1CC1', 'Clc1nncnc1[N-]C(=O)c1ccc(C#N)cc1', 'COc1ncnc(N)c1OC[C@@H]1C[C@]12CCc1ccccc12', 'Clc1ccc(C2S[C@@H](CO[C@H]3OCc4ccccc4O3)C2)cc1', 'O=[N+]([O-])c1cccc2c1C[C@H]([NH2+]Cc1ccccc1Cl)C2', 'Clc1ccccc1-c1cnnn1[C@@H]1CC[C@](O)(C(F)(F)F)C1', 'C[C@@H](c1ncc(C(=O)[O-])n1C1CC1)c1occc1COC(F)(F)F', 'Clc1cccc2ccc(COCCNC(=O)CC[C@@H]3CCC[NH+](CC4CC4)C3)nc12', 'OC[C@@]1([NH3+])CC[C@@H](CNc2ccccc2Br)C1', 'Oc1cc(O)cc2ncnc(N[C@@H](CO)[C@H](O)C(F)(F)F)c12', 'Clc1ccc([C@@H]([NH2+]C[C@H]2CCCO2)N2Cc3ccccc3C2)nc1Cl', 'Cc1cn([C@H](C)CNC(=O)C[C@H](O)c2ccoc2)n1C', 'ClCc1cn(C[C@@H]2CCCO2)c(C(=O)[O-])c1', 'Nc1[nH]nc(C2CC[NH+](Cc3ccccc3C(F)(F)F)CC2)n1', 'ClCc1noc2c1C[C@@H](CO)C2', 'Oc1cc(C[NH2+][C@@H]2CCCc3ccccc32)c(F)cc1Br', 'O=[N+]([O-])c1ccc2nccnc2c1C[NH2+]C[C@H]1CCCO1', 'Clc1ccccc1-c1csc([C@@H]2CCCN(S(=O)(=O)CCN3CCOCC3)C2)[nH+]1', 'Clc1ccccc1OCc1ccn([C@H]2CCS(=O)(=O)C2)[nH+]1', 'O=C([O-])[C@@H](c1ccccc1)[C@H]1CCCC[NH+]1Cc1ccc(Cl)cc1', 'C=C1c2ccccc2C(=O)O[C@H]1CC(=O)N1CCC2(C1)OCCO2', 'COC[C@](C)(O)CNC(=O)N(C)[C@@H](C)C(C)(C)C', 'ClC[C@@H]1C(=O)CCC(C2CC2)C1', 'O=[N+]([O-])c1ccn(-c2cc(C[NH3+])ncc2F)c1', 'C[C@@H]([NH2+][C@H](CO)c1ccccc1Cl)c1cccc(F)c1', 'O=C(C[NH+]1CCN(c2cccc(F)c2)CC1)Nc1ncc(-c2nccn2C(F)F)cn1', 'O=C(/C([O-])(c1ccc(F)cc1)c1cnc(Cl)cc1)[C@@H]1SCCS1', 'Clc1cccc(S[C@@H](C(=O)CCC2CC[NH+](C3CCOCC3)CC2)c2ccccc2)c1', 'COCC[C@@H](C)C(=O)N[C@@H]1CC[NH+](Cc2ccccc2)C1', 'CN(C[C@H](C[NH3+])c1ccc([N+](=O)[O-])cc1)C(C)(C)C(=O)NC1CC1', 'Clc1ccc(COC2CC[NH+](C[C@@H]3CCCO3)CC2)cc1', 'NS(=O)(=O)N(C1CCCCC1)[C@@H]1CCC(=O)O1', 'Clc1cc(Cl)ncc1[C@@H]1CCOC2(CCCC2)C1', 'O[C@@H](c1cc(I)ccc1F)[C@@H]1CN(C(F)(F)F)CCO1', 'O=[N+]([O-])c1c(Br)c(Br)cc([O-])c1[N+](=O)[O-]', 'Clc1ccsc1N[C@H](C1CCOCC1)S(=O)(=O)c1cccc2nonc12', 'COc1nc2c(c(=O)[nH]1)[C@@H](c1c(F)cccc1Cl)[C@@H](C(=O)Nc1ccccc1Br)CS2', 'Clc1cc(Br)cnc1[S-]', 'ClCc1cc(Cl)cc(C(=O)NCC[NH+]2CCOCC2)c1', 'Clc1ccccc1OCc1csc([C@H]2CCC[NH2+]C2)n1', 'Clc1ccc(C2SCCC[S@@]2=O)cc1', 'O=C(C1=c2[nH]ccc2=[NH+]C1)N1CC[C@@H](Nc2cccc(Cl)c2)C1', 'ClCc1nnsc1Br', 'OC[C@@H](CC1CCOCC1)NNC(=O)[C@@H]1CCCO1', 'COc1nc2c3c(sc2n1CCCN(C)C(=O)C[S@@](=O)C)CC3', 'O=C(/C(c1cccs1)(Br)F)C(F)(F)F', 'Nc1cn[nH]c1CSc1n[nH]c(=O)[nH]1', 'O=C([C@H]1c2cc(S(=O)(=O)[O-])ccc2N=C1N1CCOCC1)NNc1ccccc1C(F)(F)F', 'O=[N+]([O-])c1c(Br)scc1[C@@H]1Nc2ccc(Br)cc2N1', 'O=C(CN(CC[NH+]1CCCC1)C1CCCCC1)N1CCCC1', 'OC[C@@]1(OC2CCCCC2)C[C@H]1c1cnc[nH]1', 'Clc1ccccc1-c1n[nH]cc1C[NH+]1CCC(C2CCN(C(=O)c3ccco3)CC2)C1', 'NS(=O)(=O)/C(=O)c1cccc2c1[C@H]([C@@H]1CCCCN1C(=O)c1ccco1)N2', 'O=C(Cc1nc[nH]n1)Nc1c(Br)cc(Br)s1', 'N=CC1(Sc2nc(Cl)ncn2)CCCCC1', 'Oc1ccc([N+](=O)[O-])c(CC[NH2+]C2CC2)c1Cl', 'O=C([O-])c1nnc([C@H](c2cccnc2)C2CC2)s1', 'Clc1ccc(-c2n[nH]cc2C[NH+]2CCc3[nH]c4ccccc4c3C2)cc1', 'O=[N+]([O-])c1c(F)cccc1N1CC[C@@H](C[NH+](Cc2ccccc2)C2CC2)C1', 'O=C(c1c[nH]cn1)[C@H]1[C@H](c2cccc(Cl)c2)CCC1=O', 'O=C(/C=C/O[C@@H]1COc2ccccc2O1)C1=NN(c2ccccc2)SC1=O', 'Nc1[nH]ncc1N[C@H]1CCN=C1Cc1nnc2n1CCC2', 'O=C(CCS)/C(=N\\c1ccccc1)Sc1nncn1CC(=O)[C@H](O)c1cccc(Br)c1', 'Clc1cc(/N=C/c2sc3ccccc3c2C[NH+]2CCCC2)cc(O)c1O', 'ClC[C@H]([NH2+]Cc1ccccc1Br)c1ccsc1', 'Cc1cccc2c1[C@@H](Cc1cccnc1)C2CC[NH3+]', 'Clc1cc(Cl)nnc1N[C@@H]1CCOC2(CCCCC2)C1', 'Clc1cc(C[NH2+][C@@H]2CSc3ccccc32)ccc1Br', 'O=C(CNC(=O)CN1C(=O)[C@@H]2CC=CC[C@H]2C1)Cc1cc2ncccc2s1', 'Oc1cccc([C@H](CNS(=O)(=O)c2ccc(Cl)cc2)[NH+]2CCCCC2)c1', 'Oc1cc(O[C@@H](C[NH+]2CCCCC2)c2ccccc2)ccc1OC[C@@H]1CCCO1', 'ClC(/N=C1/CSC[C@@H]1N1CCCC1)c1ccc(SC(F)F)cc1', 'O=C(CS(=O)(=O)CCN1CCC([NH+](CC)C2CC2)CC1)c1ccc(F)cc1F', 'Clc1ccc(Br)cc1O[C@@H]1CCOC2(CCC2)O1', 'ClC[C@@H]1C[C@H]1c1cc(I)c(Cl)cc1Cl', 'Clc1cccc([S@@]CC[C@@H]2CCC[NH2+]C2)c1', 'Oc1cc(F)ncc1C[NH+](Cc1ccco1)Cc1ccco1', 'Oc1ccccc1[C@H]([NH2+]C[C@@H]1CC2(CCCC2)O1)c1ccccc1', 'Oc1ccc([N+](=O)C2CCCC2)cc1OC[C@@H]1CCC[C@@H]1O', 'Clc1cccc(-c2cnc(C[NH+]3CCN(Cc4ccc(F)cc4F)C3)o2)c1', 'Clc1cccc2nc(SC[C@H]3CSCCO3)nn12', 'Clc1ccc([C@@H]([NH2+][C@H]2CCCc3ccc(F)cc32)C2CC2)cc1', 'Clc1cccc(Br)c1S(=O)(=O)C1=c2ccccc2=[NH+]C1', 'ClCc1csc2nc([C@@H]3[C@@H](Cc4ccccc4)C(=O)N3)sc12', 'O[C@@H]1[C@@H](Cc2ccccc2)N/C1=C[C@@H]2c1ccccc1C2=O', 'O=C(C[NH+]1CCC[C@H]1COc1ccc2n[nH]c(=O)cc2c1)N1CCCCC1', 'ClC[C@H]1CN2CCC[C@@H]2CO1', 'NS(=O)(=O)/C=c1/C(C#N)c2cc(Cl)c(C(=O)N3CCOCC3)cc2nc1/C=N/C(=O)c1ccccc1O', 'Oc1cccc([C@H]([NH2+]C2CCCC2)C2CC2)c1', 'Clc1ccccc1Nc1nn[n-]c1C(=O)[O-]', 'Clc1ccc(Br)cc1SC1CC[NH2+]CC1', 'Oc1cccc(SC2[C@H]3CCCC[C@H]23)n1', 'Oc1cc(C[NH2+][C@@H]2CCc3ccccc32)ccc1Br', 'O=[N+]([O-])c1cc(F)ccc1C[NH+]1CCC(c2ccccc2)=N1', 'Oc1cccc(N2CC[NH2+]CC2)c1O', 'O=[N+]([O-])c1c([N+](=O)[O-])cc(I)cc1N1[C@@H]2CC[C@H]1C[C@@H]2c1ccc(F)cc1', 'COc1nn(C)cc1C(=O)N1CC[C@H](C#N)C[C@@H]1c1ccc(C)o1', 'O=C(c1cn[nH]n1)N1CCC[C@H](c2[nH+]ccn2Cc2cccc(F)c2F)C1', 'Oc1cc(-n2nnnc2O)cs1', 'OCSc1cn[nH]c1C#N', 'Clc1ccccc1C[NH+](C[C@]1(Cc2ccccc2)O[C@H]1CCO)C1CC1', 'Clc1cc(/C=N\\O[C@@H]2CCC[C@@H]2[C@@H]2CCCO2)ccc1O', 'Clc1cc(N2CCc3nc(CSc4ncccc4O)[nH]c3N2)cs1', 'Clc1ccsc1-c1nnon1', 'Clc1nnnc(Br)c1I', 'Clc1nnnc(Br)c1I', 'Clc1cc(COC2=N[C@@H]3CS(=O)(=O)C[C@@H]3N(c3ccncc3)C2)ccc1Cl', 'Oc1cccc(NC2CCC([NH2+]CCc3c[nH]nc3C)CC2)c1', 'Clc1ccc(-c2nonc2[S-])s1', 'Clc1ccc(-c2nonc2[S-])s1', 'Oc1ccccc1[C@@H]([NH+]c1c(Br)cccc1Cl)[C@@H]1CCCO1', 'N#C/C(=C\\c1cnc(-c2ccco2)cc1O)[C@@H]1CCCO1', 'Oc1ccc([N+](=O)[O-])c(/[NH+]=C/c2ccc(Cl)s2)c1', 'Oc1cc(F)c(S(=O)(=O)[N-]c2ccc3c(c2)OCCO3)cc1C(=O)[O-]', 'Nc1cc([O-])nc([C@@H]2CCCN(c3ncnc4ccsc34)C2)n1', 'Oc1ccc(/N=c2/[nH]cc[nH]2)nc1', 'O=C(CC[NH+]1CCC[C@@H]1c1ccco1)Nc1ccccc1[N+](=O)[O-]', 'Clc1cccc(C2=NOc3cc4ccccc4oc3=[NH+]2)c1', 'Clc1cc(Cl)nn1C[NH+]1CCN(c2ccccc2)CC1', 'Oc1cccc(CC[C@H]2CCCC[NH2+]2)c1', 'CCC(=O)N1CCC2(CC1)OC(C(=O)N1CCC(C)CC1)CC2', 'N=CC[C@H]1C(=O)NC(=O)C[C@@]1(O)CSc1ccccc1', 'Clc1ccsc1[C@@H]1CSCCS1', 'Clc1ccc(-c2cc(/[NH+]=C/c3ccco3)nc([S@](=O)c3ccccc3)n2)cc1', 'N[C@@H](NC[C@H](c1ccco1)[NH+](C)C)c1ccccc1', 'OC[C@@]1(c2cccs2)NC(=S)N1Cc1cccs1', 'Clc1nnnn1/C(=C\\c1nc[nH]n1)c1ccc(I)cc1', 'Clc1ccc(CCC[NH+]2CCCC[C@H]2c2ccccc2)s1', 'Oc1ccc(/C=C(\\[O-])/C([O-])=N\\c2ncccn2)c(O)c1', 'Clc1ccc(Br)cc1CO/[NH+]=C(\\N)C#N', 'OC[C@@H]1CN(Cc2cccc([S@@](=O)C(F)F)c2)CCS1', 'O=[N+]([O-])c1cc(Cl)ncc1[C@@H]1CCOC2(CCCCC2)C1', 'ClCc1cc(CO)oc1[O-]', 'Clc1cccc(F)c1C[NH+]1CC[C@H](Nc2ccccc2)C1', 'Clc1cc(/C=N\\O[C@H]2c3ccccc3O2)ccc1[O-]', 'Oc1cccc(N2CC[NH+](Cc3ccc(Cl)o3)CC2)c1', 'O=[N+]([O-])c1ccc([C@H]2CSC[C@@H](Cc3cccs3)O2)c(Cl)c1', 'Clc1ccccc1COCC[NH+]1C[C@@H](C)[C@H](Oc2ccccc2C(F)(F)F)C1', 'Clc1ccc(-c2nonc2C/[NH+]=C2\\SC3=C(CCCC3=O)NC2=O)cc1', 'Clc1cccc([C@@H]([NH2+]Cc2nccs2)C2CC2)c1', 'O[C@@H](Sc1cnn(CCO)c1)C(F)(F)Cl', 'Oc1ccccc1[C@H](N[C@H]1C[C@@H](O)[C@@H](O)c2ccccc21)c1nnc2cccnn12', 'Clc1nncc(N[C@H]2CCCC[C@@H]2n2cnnc2)c1Cl', 'Oc1cccc2nnc3cc([C@@H](CCO)c4ccccn4)nc([O-])c3c12', 'Clc1ccsc1C[N-]C(=O)c1ccc(Cl)s1', 'O=C(C[NH+]1CCN(Cc2cccnc2)C1)Nc1cnccn1', 'Clc1nnnc(Br)n1', 'O=C(C1=C(NC2=NN=C(SCc3ccccc3F)N=N2)[C@@H]2CS(=O)(=O)C=C12)Nc1ccc(Br)cc1', 'Nc1c(O[C@@H]2CCOC3(CCOCC3)C2)nc(N2CCOCC2)n1C1CC1', 'C=C1NC(=[NH2+])[C@@H]1/C=N/NC(=O)C(=O)N/N=C/c1ccc(C)cc1', 'O=C(Cc1noc([O-])n1)N[C@H]1c2ccccc2C[C@@H]1O', 'CN=Nc1nc(CNCc2ccccc2O[C@@H]2CCC[C@@H]2OCC(F)F)n[nH]1', 'Oc1ccc(/C=C(/SCc2cccc(C#N)c2)NN[C@H]2CCCO2)c([N+](=O)[O-])c1', 'O[C@@H]1c2cc(Cl)ccc2N[C@H]1CC(F)(F)F', 'O=C([O-])[C@@H]([NH3+])c1csc(Cc2cccc(F)c2)n1', 'Clc1cccc(Br)c1S[C@H]1CCOC[C@@H]1C(=O)[O-]', 'Cc1sncc1SCC(=O)N1CCN(c2cccc[nH+]2)CC1', 'C[C@@H](c1noc(-c2ccc(O)cc2)n1)N1CC[NH+](C[C@@H]2CCCO2)CC1', 'O=C(/C=C/Oc1ccccc1F)N[C@H]1[C@H](C[NH+]2CCCCC2)C[C@H]1c1ccc(F)cc1', 'O=C(c1cn[nH]c1)[C@H]1CC[C@H](C(=O)[O-])C1', 'Clc1ccccc1N1Nc2c[nH+]ccc2C[C@@H]1c1cc(Cl)ccc1Cl', 'Clc1nncn1[C@@H]1CCOC2(CCCCC2)C1', 'Clc1nnnn1C1CCN(c2cccc[nH+]2)CC1', 'Clc1ccc(-c2cc[nH]n2)nc1[C@H]1COCCO1', 'O=C(c1cnn(C(=O)N[C@@H]2CS(=O)(=O)C[C@@H]2C(=O)[O-])c1)c1ccccc1', 'ClCc1csc(C[NH2+]C2CCCC2)n1', 'ClC(F)(CO)[NH2+]Cc1ccc(Cl)cc1', 'CCCS(=O)(=O)N1CCN(C(=O)[C@H]2CCCC[C@H]2[NH3+])CC1', 'Oc1cccc([C@H]2O[C@@H](c3ccccc3)S/C2=N\\C2=C/C=C\\c3nc4ccccc4n3-2)c1', 'Nc1[nH]nc(N2CC[NH+](Cc3cnn4ccnnc34)CC2)c1-c1ccc(Cl)cc1', 'O=C([C@H]1c2cc([N+](=O)[O-])ccc2O1)N1CCSC[C@H]1C1OCCO1', 'C=C1[C@H](C(=O)Nc2cc(Cl)c(C)cc2C)S[C@@H](C)C[C@H]1C', 'Clc1ccc([C@@H]([O-])c2ccc(OCC[NH+]3CCC(CO)CC3)cc2)cc1', 'Clc1nnnn1c1c(Cl)cccc1N1CC[NH2+]CC1', 'ClC[C@H](Br)NC(=O)CC[C@H]1CCCO1', 'COc1ncc(Br)cc1C(=O)N[C@@H]1C[C@@H](C)N=C(C#N)[C@H]1c1ccccc1Cl', 'OC[C@@H](Nc1ncccc1C(F)(F)F)N1CC[C@@H](C)[C@@H](O)C1', 'O=[N+]([O-])c1c(F)cc2n1CCC[C@@H]2COC1CC1', 'O=C([O-])/C=C/[C@]1(CN2C(=O)Cc3ccccc32)CCC1=O', 'Clc1nnc(Cl)[nH]c1=S', 'N[C@@H]1NC(=O)/C(=C/c2ccco2)S1', 'Oc1ccc(Oc2cnc(C[NH+]3CCCCC3)s2)cc1', 'Clc1ccc(-c2nonc2C[C@@H]2CCCC[NH2+]2)cc1', 'Clc1ccsc1C[N-]c1ccc(C(F)(F)F)nc1', 'N[C@@H]1[C@H](C2CC2)N=C(NCc2cc(Cl)ccc2-n2ccnc2)C1', 'Clc1nnnc(-n2nc([O-])nc2-c2ccc(Cl)s2)n1', 'Clc1ccc(Br)c(Cl)c1N1CCC[C@H]([NH3+])C1', 'Clc1ccc(C[NH+]2[C@H]3CCCC[C@H]3C[C@@H]2C2CCCC2)cc1', 'O=C(C1=c2ccccc2=[NH+][C@H]1N1CCOCC1)N1CCc2nccnc2C1', 'Clc1ccsc1N/C=C1\\NC=CCC1', 'O=C(COc1ccc(Cl)cc1)NCC1([NH+]2CCCC2)CC(C)CC1', 'N=CC[C@@H]1CCCN1c1ncc(C(=O)[O-])cc1Cl', 'C[C@@H](Cn1c(C[NH+](C)CCCN2CCOCC2)noc1=O)c1ccccc1', 'Clc1ccc(Br)cc1SCCCC1[NH+]=c2ccccc2=[NH+]1', 'Nc1[nH]nc(S[C@@H]2CCOC2)c1[N+](=O)[O-]', 'Clc1cccc(N2N=C(CC3=c4ccccc4=[NH+]C3)CC2)c1', 'Oc1cccc(Br)c1C1CC[NH+](Cc2ccc(C#N)cc2F)CC1', 'Clc1ccc(C2CC[NH+](C[C@@H](O)c3cnccn3)CC2)cc1', 'NS(=O)(=O)/C(C#N)=C1\\C=CC(=O)N[C@@H]1c1ccccc1', 'O=C(COc1cc(C[NH+]2CCOCC2)ccc1Br)NC[C@H](c1ccccc1)[NH+]1CCCC1', 'N#C[C@@]1(c2cc[nH]c2)CN(CC(F)(F)F)c2ccccc21', 'Clc1ccccc1Nc1ncc2c(c1C(=O)[O-])C[NH+](Cc1cccnc1)C2', 'Clc1ccc(C[NH+](CCc2ccco2)[C@@H]2CCCc3cc(Cl)ccc32)cc1F', 'Oc1ccccc1[C@@H]1C[C@@H](C[NH2+][C@H]2CCc3ccccc32)c2ccccc2S1', 'O=C([O-])CSCC[C@@H]1CCCO1', 'Oc1cccc(O[C@H](C(=O)[O-])C[NH+]2CCN(Cc3ccccc3)CC2)c1', 'Clc1ccc([C@@H]([C@@H](C(=O)[O-])Cn2cccc2)C(F)(F)F)cc1Cl', 'Clc1ccc(Br)cc1-n1nncc1C[NH+]1CCCCC1', 'Oc1ccccc1OSC[C@H]([NH3+])CO', 'O=C(Cc1n[nH]n2c[nH+]cc12)Nc1cccnc1', 'Oc1ccc(/N=c2/sc3c(s2)CCN3C(F)(F)F)c(F)c1', 'O=[N+]([O-])c1ccncc1CC[NH+]1CCCCC1', 'ClC(/N=C1/SC2=C(CCCC2)SC1=S)c1ccccc1', 'COc1nc2nc(N(C[C@@H]3CCCO3)C(C)C)c(C)nc1-2', 'OC[C@@H]([NH2+]Cc1ccccc1Br)C(F)(F)F', 'Clc1cc(NC[C@@]2(C(=O)[O-])CCCN2S(=O)(=O)c2ccccc2)sc1C1CC1', 'NS(=O)(=O)Cc1cc(NCC[NH+]2CCCCC2)ccn1', 'Clc1ccsc1C[N-]c1ccc2[nH]ccc2c1', 'Clc1nnnn1/C=C/C(F)(F)F', 'Oc1ccc([C@H](NNC2CC[NH+](Cc3c[nH]c4ncccc34)C2)c2ccccc2)cc1', 'Clc1ccc(Br)cc1C[NH+]1CCC[C@H]1COc1ccc(Cl)cc1', 'Clc1ccc(-c2cc(=O)n3nc(-c4ccco4)s3c2)cc1', 'Clc1ccc(Br)cc1N(CCS(=O)(=O)c1ccc(Cl)c(Cl)c1)[C@H]1CCS1(=O)[O-]', 'OC[C@H]1ON=C2CCCN2/C1=C/c1ccnc([N+](=O)[O-])c1', 'C=CCC(C)(C)C[C@](C)(O)CC', 'Clc1cccc(/C=N/c2ccc[n+]c2)c1', 'Oc1ccc(O[C@H](Cc2ccc[nH]c2=O)[C@@H]2CC[NH2+]C2)cc1', 'Oc1ccccc1[C@@H]1OCC[NH+](CN2C(=O)c3ccccc3C2=O)C1', 'Clc1cc(F)cc(F)c1N1CC[NH+](C[C@H](O)CN2Cc3ccccc3C2)CC1', 'COC[C@@H]1[C@H](CC2CCCCC2)NC(C)=C1C(C)=O', 'Clc1ccc(CO)cc1SCC[C@H]1CCC[NH+](C(=O)Cc2ccccc2)C1', 'Oc1cc(F)cc([C@@]2(NC(=O)c3ccc[nH]c3=O)N3CCCC[C@H]32)c1', 'O=[N+]([O-])c1ccc(CNC[C@@H]2CCC[NH2+]2)cc1', 'Oc1cccc(OC2CCN(C/C=N\\CC[NH+]3CCC(Cc4ccccc4F)CC3)C2)c1', 'O=[N+]([O-])c1c([O-])ncnc1[C@@H]1CCOC1', 'O=C(C[NH+]1CCC[C@@H]1c1ccccc1)Nc1nnc(C2CCCCC2)s1', 'Clc1nncnc1[N-]S(=O)(=O)c1cc(F)cc(F)c1', 'COC(=O)[C@H](C[NH+]1CCCCC1)c1ccc(F)cc1', 'NS(=O)(=O)C[C@@H](SCC(=O)c1ccsc1)C(=O)[O-]', 'Oc1cccc([C@H](COC2CC[NH2+]CC2)N2CCOCC2)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1N[C@H]1CCCS1', 'Oc1cccc(S[C@@H]2C[C@@H](O)[C@@H](O)[C@@H]2O)c1', 'NS(=O)(=O)N1CCN([C@H]2CCOC3(CCCC3)C2)CC1', 'Cc1ncsc1C[NH2+][C@@H](C)c1ccccc1O', 'CCn1cc([C@@H]2C[C@@H]2C(=O)O[C@H](C)C(=O)NCC[C@@H]2CCCO2)s1', 'O=C([O-])/C=C/C=C/C=C/C1=CCCCC1', 'OC[C@H]([NH2+]Cc1ccccc1)c1cc(Cl)sc1Cl', 'Clc1ccccc1OC1C=C[C@H](OC2CCCCC2)[NH+]1', 'Oc1ccccc1OSCc1n[nH]c2nc([O-])oc12', 'O=C([C@H](Sc1nnnn1Cc1ccccc1)[C@H]1CCCO1)N1C[C@H](CO)CCO1', 'Clc1ccc(CC[NH2+][C@H]2CCC[C@@H](O)C2)cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])c(Br)sc1[O-]', 'Clc1cccc2c(N)cc(C[C@@H]3CN4S(=O)(=O)CCC[C@@H]4O3)cc12', 'Oc1ccccc1C[NH+]1CCC[C@@H]1CN1CCCC1=O', 'COC[C@H]1CCC[NH+](Cc2cccc(OC)c2OC)C1', 'Cc1sc2ccccc2c1C(=O)N(C)[C@@H]1CCC[C@]2(CCOC2)C1', 'O=C(C1CCCCN1c1ccc2c(c1)COC2)NN1CCC([NH+]2CCCC2)CC1', 'Clc1ccc(COC[C@@H](O)C[NH+]2CCCCC2)cc1', 'Clc1ccc(Br)c(OC[C@@H]2C=c3ccccc3=[NH+]2)c1', 'Clc1nncnc1[N-]OCC(=O)c1cccs1', 'O=[N+]([O-])c1ccncc1C[NH2+][C@@H]1CCN(C[C@@H]2CCCO2)C1', 'Clc1ccc(-c2nnnn2[C@@H]2CCC[NH+](Cc3cc(F)ccc3F)C2)cc1', 'N=C1/C(=C\\C=C\\c2cccc(Cl)c2)C[NH2+]C1', 'O=C(/N=c1c(N)nc2c(Cl)cccc12)c1ccc[nH+]c1', 'C[C@@H](C[NH+](C)CC(=O)NCc1cccc(F)c1)NS(C)(=O)=O', 'Cc1nonc1C[NH+]1CCC(NC(=O)c2ccc(C[NH3+])cc2)CC1', 'N#CCS(=O)(=O)c1ccccc1C[NH+]1CCc2ccccc2C1', 'OC[C@H]1NC(=S)c2cccnc2C1=C(c1cccs1)N1CCCC[C@H]1Cc1ccccc1', 'NS(=O)(=O)C[C@@H]([NH+]1CCc2ccccc2C1)c1ccccc1', 'CC[C@@H]1O[C@H](C(=O)N2CCc3c([nH]c4ccccc34)N2C)C(=O)N1', 'O=C(C[NH+]1CCCC1)NC[C@@H](O)c1ccc(Cl)cc1', 'Clc1cccc2[nH]nc(S/C=C/C[NH3+])c12', 'N=Cc1nonc1[N-]S(=O)(=O)c1ccc2c(c1)OCCO2', 'Clc1nnc(S(=O)(=O)NC[C@@H]2CCO[C@@H]2CO)cc1F', 'CNS(=O)(=O)[C@@H]1CC[NH+](C2CCN(Cc3cccc4ccccc34)CC2)C1', 'Clc1ccc(Br)c(-c2n[nH]c([C@@H]3CCCN(/C=C/CCC[NH+]4CCCC4)C3)n2)c1', 'O=C(CS(=O)(=O)[C@@H]1CCSC1)Nc1cc(C(F)(F)F)cs1', 'O=C(C1=CC[C@@H](c2ccccc2)/C(=C\\c2cccs2)C1)N1CCC(O)[C@H](O)C1', 'ClC[C@H]1OC[C@H]2CCCC[C@@H]2C1', 'OC[C@@]1([NH3+])Oc2ccccc21', 'Clc1cccc([C@@H]2[NH2+]CC[C@@H]2[C@@H]2CCCN2C(=O)C2CC2)c1', 'Clc1ccc(Br)cc1NCC[NH+]1CCCC1', 'NS(=O)(=O)/N=C(\\[O-])c1n[nH]c(-c2ccoc2)n1', 'Clc1cc(Cl)nc2[nH]nc(C(=O)C[S@](=O)CC3CC3)c12', 'Clc1ccc(C[NH+](Cc2ccc(F)cc2)[C@H]2CCOC2)cn1', 'Clc1cc(C[C@@H]2CCCN(S(=O)(=O)C[C@@H]3C=C(CN4CCOCC4)CO3)[C@@H]2c2ccccc2)ncn1', 'Clc1ccccc1OC1C[NH2+]CC1', 'Clc1nnc(/C=C/CCO)c([O-])n1', 'Clc1ccc(Br)cc1/C=N/N1CC[NH2+]CC1', 'ClC([O-])CC[C@@H]1CCCC[C@@H]1n1ccnc1', 'Clc1ccc(-c2nn([C@@H]3CC[C@@H](O)[C@@H](CO)[C@@H]3O)cc2F)cc1', 'NS(=O)(=O)Cc1noc(S(=O)(=O)NNC(=O)C(c2ccccc2)c2nc[nH]n2)n1', 'Clc1nnc(Br)c(Br)c1C(=O)[O-]', 'CN(c1ccn([C@@H]2CCS(=O)(=O)C2)c1)c1nc(C2CC2)no1', 'O=[N+]([O-])c1cc([C@H]2CC(=O)N(c3ccccc3)c3ccccc3C2)ccc1N1CCOCC1', 'Clc1ccccc1C(CCCO)C[NH2+]C[C@@H]1CCCO1', 'O=C(/N=c1/[nH]c2ccccc2cc1C(F)(F)F)C(=O)N1CCC[C@@H](c2nnc[nH]2)C1', 'Clc1cccc2[nH]nc(N[C@H]3CCCC[C@@H]3[NH2+]C[C@@H](O)c3ccccn3)c12', 'Clc1ccc(C[NH+](CC[C@@H]2CCCO2)Cn2cccn2)nc1', 'C=C([O-])/C=C(\\C)NC(C)(C)C', 'Clc1nncn1/C(CO)C[C@H]1CCC[C@@H]1O', 'ClC[C@@H]1COC[C@@H]1C1CCC1', 'O[C@@H]1[C@@H](OCC(=O)OC2CCCCC2)[C@@H]2CCCC[C@@H]21', 'O[C@@H]1[C@@H](C[C@@H]2CCCO2)C(=O)C[C@@H]1c1ccc2c(c1)OCO2', 'ClC[C@H](O)[C@@H](n1cncn1)C1CC1', 'Clc1ccc(Br)cc1C[NH+](Cc1cc(O)ccc1C)[C@H]1CCSC1', 'OC[C@@H]1N=N/C(=N\\CCCc2ccco2)C1', 'Clc1nncn1[C@@H]1CCCC[C@H]1C(=O)[O-]', 'Clc1ccc([C@@H](Cl)[C@H]2C[C@@H]2CC(=O)[O-])cc1', 'Cc1sncc1-c1c(C)nc2[nH]c(C(C)C)nc2c1[O-]', 'Clc1cc(Br)ccc1S[C@@H]1CCOC([O-])C1', 'Clc1ccc(-c2cc[nH]n2)nc1C[NH2+]CC[C@H]1OCCO1', 'ClC([O-])[C@@]1(O)Cc2ccccc21', 'CCCC[NH+](C)CC1CCN(c2nccnc2C#N)CC1', 'Clc1ccccc1C[NH+](C1CC1)[C@@H]1CCC[C@H](C(F)(F)F)C1', 'C[C@@H]([NH2+]CC1([NH+]2CCOCC2)CCCCC1)c1nc2ccc(-c3ccccc3)cc2[nH]1', 'ClCc1ccc2c(c1)CCC[C@@H]2[NH2+][C@@H]1CCSc2ccccc21', 'O=C(CSCC(N)=N/O)[C@H]1CCC(=O)O1', 'Oc1ccc([N+](=O)[O-])c(/C=C2\\S[C@@H]3CS(=O)(=O)C[C@@H]3N2C(=O)c2ccco2)c1', 'CN=C1/C(=N/C(=O)[C@@H](C#N)c2ccc(Cl)cc2)S/C1=N\\c1cccc(F)c1', 'Clc1cccc(S(=O)(=O)/N=C(\\[O-])[C@@H]2CCOC3(CCCC3)C2)c1', 'C[C@@H](c1ncnn1C)C(=O)[O-]', 'O=[N+]([O-])c1cc(Cl)cc(N2CCC([NH2+]CC[C@@H]3COCCO3)CC2)c1', 'O=[N+]([O-])c1c(Br)cncc1Sc1ccc(C[NH3+])s1', 'Oc1ccc(F)c(Cl)c1OC1CC[NH2+]CC1', 'Cc1cn[nH]c1C(=O)N[C@H]1CC[C@@](CO)(C(C)(C)C)C1', 'Clc1ccccc1CSC1=NC2=CC=N[C@@H]2S1', 'Clc1ccc(Br)c(NCC[NH+]2CCCC2)c1', 'ClC(c1cc(Cl)on1)c1cc(I)cc(O)c1', 'O[C@@H]([C@@H]1CCC[NH+]1Cc1c(Cl)cccc1Cl)NCc1ccccc1', 'N#C[C@@]1(C2CC2)Nc2cccnc2N1', 'Oc1ccc([C@H]([NH2+]C[C@H]2CCCO2)[C@@H]2CCSc3ccccc32)cc1', 'Clc1cccc(Nc2cn[n+]([O-])cc2O[C@@H]2CCCCS2)c1', 'ClC[C@H]1CCOC2(CCCCC2)C1', 'Clc1cc(CC2(C[NH2+]CC[C@H]3CCCCO3)CCC2)ncn1', 'OC[C@H](C[C@@H]1CCC[NH2+]1)c1nc(-c2ccccc2)no1', 'NS(=O)(=O)Nc1cc(F)ccc1C(=O)NC[C@H](c1ccco1)[NH+]1CCCC1', 'O=C(/C=C1CC(C)(C)[NH+](Cc2ccccc2)C1=O)c1cc(Cl)c(Cl)cc1Cl', 'ClCc1cc(Br)cnc1N1CC[C@H]([NH+](C)Cc2ccccc2)C1', 'OCc1ncccc1CNC[C@H]1CC[NH+]=CC1', 'Clc1ccccc1NCc1cc[nH+]cc1-[nH+]1cc[nH+]c1', 'O[C@@H](c1csnc1)[C@H]1CC(=O)N(c2cc(Cl)ccc2C#N)C1', 'NS(=O)(=O)N(CC(=O)Nc1ccc(F)c(Cl)c1)[C@@H]1CCC[NH+](C2CC2)C1', 'Clc1ccc(C2(NCC(=O)[O-])C[C@@H]2C2CCOCC2)cc1', 'O=[N+]([O-])c1c(F)cc([O-])cn1', 'O=C(CNc1cc2n(n1)CC[NH2+]C2)c1cccs1', 'N#C/C(=C1\\N=C[C@H](S/C=N/S(=O)(=O)c2ccccc2)S1)c1ccccc1', 'O=C([C@H](CC(F)(F)F)N1CCCS1(=O)[O-])N1CCc2ccccc21', 'Clc1ccc(-c2n[nH]c(NC[C@H]3CCCC[NH2+]3)n2)cc1', 'Nc1[nH]nc2c1[C@@H]1CCC[C@H]1C2', 'Oc1ccc(O[C@H]2C=[NH+][C@@H](COc3cc(C(F)(F)F)ccc3Cl)C2)cc1', 'Oc1cc(-n2nnnc2S[C@@H]2CCCC[C@@H]2C[NH2+]Cc2ccccc2)ccc1Cl', 'Clc1ccc([C@@H]([C@@H]2CCOC3(CCCCC3)C2)OC[C@H](O)CO)cc1', 'N#CC[C@]1(O)CCCN(C(=O)c2cn(Cc3ccccc3)n[nH+]2)C1', 'Oc1ccc(O[C@@H]2CCC[NH2+]C2)cc1', 'COc1nc(C[NH+]2CCN([C@@H](C)CC(F)(F)F)CC2)c2ccccc2n1', 'NS(=O)(=O)NC[C@](O)(c1cccs1)C(Cl)(Cl)Cl', 'Clc1ccc(Br)cc1N1CCC[C@H]1C[NH+]1CCCC1', 'Clc1ccc(C[NH2+]C[C@H]2CCCc3ccc(F)cc32)o1', 'Oc1cc(OC(C(=O)[O-])c2ccc(C(C)CO)cc2)cs1', 'Clc1ccc([C@@H](CNC(=O)[C@@H]2CC(=O)N(SCc3ccco3)C2)c2cnn(-c3ccccc3)c2)o1', 'Clc1ccc(Br)c(C[NH+]2CCN([C@@H](C(N)=O)c3ccccc3)C2)c1', 'O=[N+]([O-])c1c(-c2nc(C3CC[NH2+]CC3)co2)cc(O)c2c1CCN(C(=O)[C@@H]1COc3ccccc3O1)C2', 'Nc1nc(C[C@@H]([NH2+]Cc2cnco2)c2ccc(F)cc2Cl)ccn1', 'Clc1cccc(/C=C/C[NH2+][C@@H](C(=O)[O-])C2CCCC2)c1', 'ClC[C@@H]1OCC(CN2CCC[C@@H]2C(N)=O)C1', 'O=C(c1c[nH]nc1C[NH+](CC1CC1)C1CC1)N[C@H]1CCN(c2ccc([N+](=O)[O-])cc2)C1=O', 'Clc1ccc(Br)cc1NCC[C@@H]1CCCC[NH2+]1', 'O=C([O-])[C@H]1OC(c2ccc([N+](=O)[O-])cc2)C(=O)N=C1N1CCCCC1', 'Oc1ccc(/N=c2/[nH]cc[nH]2)nc1Cl', 'Clc1ccccc1OCC[NH2+][C@@H]1CCC[C@H](S(C)(=O)=O)C1', 'OC[C@H]1[C@H]2CCCC[C@H]2[C@H]1O', 'NS(=O)(=O)/C=c1/sc[n+]([O-])c2ccccc12', 'O=C([C@H]1c2cc(C(F)(F)F)ccc2NC1=O)N1CSC[C@H]1[C@H]1CCCO1', 'Oc1cc(CC[NH+]2CCC[C@H]2c2ccco2)ccc1N1CCOCC1', 'O=C(C[NH+]1CCc2sccc2C1)Nc1cccc(F)c1', 'CCN(C[C@@H]1CCOC1)C(=O)NCc1cccc(C[NH+]2CCCC2)n1', 'NS(=O)(=O)C[C@H](O)C1CCn2cccc2C1', 'O=C(C1CCCC1)N1CC[C@H]([N+]2=CCCC2)C1', 'Clc1ccc(Br)cc1/N=[NH+]/N=C1\\C(=O)C=C(c2cccnc2)S1', 'O=C([C@H](SC1[NH+]=c2ccccc2=[NH+]1)c1ccc(Cl)cc1Cl)NC1CC1', 'Clc1ccc(Br)c([C@H]([NH2+]Cc2cccc(Cl)c2)[C@H]2CCSC2)c1', 'Oc1cc(Br)cc(/C=C2\\CCC([C@H]3C=CC=N3)N(c3ccc(Cl)cc3)C2=O)c1O', 'N#C[C@@H](c1ccccc1)N1CCC(C[NH2+]C[C@H]2CCOC2)CC1', 'Clc1ccc(Br)cc1CC[NH+]1CCCC1', 'O=[N+]([O-])c1c(-c2c[nH]c[nH+]2)[nH]c2c1cnn2-c1cccc(Br)c1', 'O=C(C[NH+]1CCC([C@H](Br)C2CC2)C1)N[C@@H]1CCCc2ccccc21', 'Oc1cc(-n2cc(C[NH+]3CCN(c4ccccc4)C3=O)CC2)ccc1Br', 'CN[C@H](CO)C1(C)CC1', 'Clc1cc(NC2(n3cccc3)SCCN2)c2ncccc2c1', 'N[C@@H]1C[C@@H]([NH3+])CN1Cc1cccc(F)c1', 'N=CCS(=O)(=O)Cc1nc2nc(C)cc([O-])n2n1', 'C[C@@H](C[C@@H]1CCCC[NH2+]1)Nc1ccccc1Br', 'Clc1nncc(S/C(=C/C(=O)[O-])COC(F)(F)F)n1', 'Clc1ccc(Br)cc1[C@@H](O)[C@@H]1CCOC2(CCC2)C1', 'Clc1cc(Cl)nc([N-]S(=S)(=O)c2ccc(F)cc2)n1', 'Clc1cccc2ccc(S[C@H](Cc3ccco3)[NH+]3CCCC3)cc12', 'COCCn1ccc(C[NH2+][C@@H]2CCCc3c(C)cc(C)cc32)c1', 'Nc1[nH]ncc1[C@@H]([N+](=O)[O-])C(F)(F)F', 'Clc1cc(Br)cc(/C=N/NCC2[NH+]c3ccccc3C2=O)c1', 'Oc1cccc(C[NH+]2CCC[C@H](Nc3ccc(Cl)cc3)C2)c1', 'Clc1ccc(C2(CO)CC2C[NH2+]Cc2ccsc2)cc1', 'Clc1cc(/N=C/CC[NH+]2CCC[C@@H]2c2ccc(C(F)(F)F)cc2)ccc1NCC1CC1', 'O[C@@H]1[C@@H](NS(=O)(=O)c2ccc3c(c2)C(=O)c2ccccc2S3)N=C2CCCC[C@@H]21', 'ClCc1nc(C[NH+]2CCN([C@H](CO)c3ccccc3)CC2)cs1', 'ClCc1n[nH]cc1C[NH+]1CCN(c2ccc([N+](=O)[O-])cc2)CC1', 'Clc1ccsc1N[C@@H]1CCOC2(CCOCC2)C1', 'Clc1cccc(SC[C@@H]2CCC[NH2+]2)c1', 'O=[N+]([O-])c1c(Cl)n2nnc(S[C@@H]3C=CC[C@@H]3CN3CCOCC3)c12', 'Clc1ccccc1NS(=O)(=O)NC[C@]1(O)C(=O)NC([O-])C1', 'Oc1ccccc1/C=C/C[NH+](Cc1ccc(Br)cc1C(F)(F)F)Sc1ncccn1', 'ClC(c1ccc([N+](=O)[O-])cc1)[C@H]1CCOC2(CCC2)C1', 'ClC[C@H]([NH2+]Cc1ccccc1)[C@@H]1CCCC[C@@H]1O', 'Oc1ccccc1[C@H]1ON=C(/C=N\\c2ccccc2Cl)[C@@H]1C(=O)[O-]', 'Clc1ccsc1[C@@H]1[C@H]2C(=O)N(c3cccc([N+](=O)[O-])c3)CC2=N1', 'Clc1ccc(-c2n[nH]cc2-c2ccc(NC[C@H](O)C[NH+]3CCCC3)cc2)c(F)c1', 'Oc1ccc([N+](=O)[O-])cc1/C=[NH+]/C(Cl)=C/Cl', 'O=C([C@H](c1ccc[nH]1)[NH+]1CCCCC1)N1CCN(c2nc3ccccc3nc2S(=O)(=O)C2CCCC2)CC1', 'Clc1ccccc1[C@H]1CCC[NH+](Cc2cccc(O)c2)C1', 'Oc1ccc(O[C@H]2C=[NH+]C\\C=C2c2cc(Cl)ccc2O)cc1', 'Cc1sncc1C(=O)O[C@@H]1[C@@H](O)[C@H](C)OC[C@@H]1OCc1ccccc1', 'Oc1cc(F)nc(S[C@H]2CC[NH2+]CC2)c1N1CCCC1', 'O=[N+]([O-])c1ccc2oc([C@H]3CCC[NH2+][C@@H]3Cc3ccc(F)cc3)cc2c1', 'ClC(=S)N1Cc2ccccc2S[C@@H]1[C@@H](O)C1CCOCC1', 'NS(=O)(=O)/C(=N\\[N+](=O)[O-])C(=O)N1c2ccccc2S[C@H]1C(C)C', 'OCS1(Cc2ccccc2)CC[NH2+]CC1', 'Clc1ccsc1C[NH2+][C@@H]1CCc2c(F)cccc21', 'O=C(c1cc(F)c(F)cc1F)N1CCC([NH2+]CC[NH+](C)Cc2ccccc2)CC1', 'Clc1cccc2nc(N[C@@H]3CCO[C@]4(CCOC4)C3)oc12', 'Oc1ccc([N+](=O)[O-])c(S[C@@H]2CC[NH+](Cc3ccccc3)C2)c1', 'O=[N+]([O-])c1ccnc2c1[C@@H](N[C@@H]1CCOC1)CS2', 'O=C([O-])[C@H](CCc1cc(Br)ccc1F)N1C[C@@H]2CC=CC[C@@H]2C1', 'Clc1ccc(Br)c([C@H]2CC[NH2+]C[C@@H]2c2cccc(Br)c2)c1', 'O=[N+]([O-])c1c(Br)cc(I)cc1O[C@@H]1C=CS(=O)(=O)C1', 'Clc1ccsc1/C=N/N=C1\\S[C@H]2CS(=O)(=O)C[C@H]2N1CC1CC1', 'CC[S@](=O)[C@@H]1CCCC[C@@H]1NS(=O)(=O)N1CCOCC1', 'Clc1nnnn1/C(c1cccs1)[C@H]1C[C@H]1c1cccc(Cl)c1Cl', 'Oc1cc(O[C@@H]2O[C@@H]3CC[C@@H]2C3)ccc1Oc1ccccc1', 'Clc1cc(CC[NH+]2CCN([C@@H]3CCOC3)CC2)ccc1Cl', 'Clc1cc(/C=Cc2sncn2)no1', 'N[C@@H](C[NH3+])C(=O)NC[C@@H]1CCCO1', 'Oc1cc(-n2nncc2[C@@H]2CSCc3ccccc32)cs1', 'O=[N+]([O-])c1c([N+](=O)[O-])c(Br)sc1N[C@@H]1C=CS(=O)(=O)C1', 'Clc1cccc(/C([O-])/N=C(\\[O-])CC[C@@H](C(=O)[O-])c2ccccc2)c1', 'Clc1nnnn1-c1cn(Br)cn1', 'Oc1cc(F)ccc1NC[C@@H]1C[C@H]2CC[C@@H]1C2', 'Clc1ccc(-c2ccccc2[N-]C2=NC([N+](=O)[O-])C3=C2C(=O)CCC3)cc1', 'Clc1nnnn1c1nc(O)c(Cl)c([O-])c1Cl', 'ClCc1csc(N2[C@H](C(=O)[O-])CCOC2)n1', 'ClC[C@H]([NH2+]Cc1cc(O)ccc1O)N1CCOCC1', 'NS(=O)(=O)C[C@H]([NH3+])c1cccs1', 'Clc1ccc(-c2nonc2)nc1O[C@@H]1CCOC2(CCCCC2)C1', 'C[C@@H]([C@@H](O)[C@@](C)(C(N)=O)[NH2+]C1CC1)c1ccccc1', 'COc1nc2snc(/C=C(Cl)C3=NOCCO3)c(=O)c2cc1CN1C[C@H](C)O[C@H](C)C1', 'Clc1cc(Br)cnc1O/N=C1\\C(=O)N[C@@H](c2cc(Cl)c(O)cc2)[C@H]1O', 'Clc1ccc(C2=NO[C@H](C[NH+]3CCCCC3)C2)cc1', 'CNc1ccc(F)c(C[NH+]2CCCC2)c1', 'Nc1nc2ncc([N-]S(=O)(=O)c3ccc(Cl)cc3)cc2cc1C[NH2+]C1CC1', 'Clc1ccc(C[NH2+][C@H]2CCC[C@H](n3ccnn3)C2)cc1', 'Clc1ccccc1[C@H]1C[C@@H](c2cc(F)cc(Br)c2)[NH+](CC2CC2)C1', 'Clc1ccccc1-c1n[nH]cc1CN1C[C@@H]2CC[C@@H](C1)C2=O', 'Oc1ccc(/C=N/OC[NH+]2CCC[C@@H]2c2nc3ccccc3s2)cc1', 'O[C@@H](C#Cc1cscn1)C1CC1', 'O=C(C[NH+]1CC(=O)N2CCC[C@@H]2C1=O)c1ccc(F)cc1', 'Clc1ccccc1Cc1nnc([C@@H]2CCCC[NH+]2Cc2cccc(C(F)(F)F)c2)o1', 'Clc1cccc(NC2=[NH+]C[C@@H]3CCCC[C@@H]3C2)c1', 'Oc1ccc(/C=N/S[C@H]2CCS(=O)(=O)C2)cn1', 'O=C(c1c[nH]c(N2CCCCC2)n1)N1CC[C@H](n2cncn2)[C@@H]1C#N', 'O=C([O-])CN1N=N[C@@H](c2ccc([N+](=O)[O-])cc2)SC1=S', 'N[C@@H](c1ncccc1F)[C@@H]1Cn2cnnc2C1', 'Oc1ccc(F)c(/N=C2\\S[C@H](c3ccco3)N=N2)c1', 'Clc1ccccc1-c1csc(NC[C@@H]2CN(Cc3cccnc3)C[NH+]2Cc2ccccc2)c1', 'Clc1ccc(C[NH+](CC[C@H]2CCCCO2)c2ncccc2C(=O)[O-])cc1', 'Oc1cc(Br)cc([NH2+][C@H]2CCc3ccc([N+](=O)[O-])cc32)c1', 'C=C1N=C(C(=O)Nc2nncs2)[NH+](Cc2ccc(OC)cc2)C1=O', 'O=[N+]([O-])c1cc(Br)c(O)c([N+](=O)[O-])c1OCCC[NH+]1CCCC1', 'O=C([O-])c1csc(C[NH2+][C@H](c2ccccc2)C2CC2)n1', 'C[C@@H](N[C@@H](C)C(=O)N(C)C)C(=O)C1=CN(C2CC2)C1', 'Clc1cccc(-c2n[n-]c(Sc3cnnnc3)n2)c1', 'ClC[C@H](Br)c1n[nH]cc1C(F)(F)F', 'O=[N+]([O-])c1cccc2c1N[C@@H](C(F)(F)F)[C@H]1CC=C[C@@H]21', 'Clc1ccccc1CCC[NH+]1CCC[C@@H]1c1cccs1', 'N#C[C@H]1NC(=O)C[C@@H](CCl)N1c1ccccc1Cl', 'O=C([C@H]1[C@H](c2ccccc2O)C=C(CO)C1=O)NC1CC[NH+](Cc2ccco2)CC1', 'Oc1cc(C(F)(F)F)ccc1/C=N/SC1=NC2=NC=NC2=N1', 'ClC(c1ccc2c(c1)CCC[C@H]2[NH2+]C[C@@H]1CCCO1)c1ccc(F)cc1', 'O=[N+]([O-])c1c(-c2nnn[n-]2)sc2cc(Br)ccc12', 'C[C@@H](c1cnn(C)c1)c1ccccc1C[NH+](C)Cc1cnc[nH]1', 'ClC(F)(CCC(F)F)NCc1cc(Cl)cs1', 'Clc1cccc2c1[C@H]1C[C@@H]1C[NH+]2Cc1ccccc1C[N+]1CC1', 'Clc1cccc(F)c1O[C@H]1C[C@H]2CC[C@@H]1C2', 'Oc1ccc([N+](=O)c2cc(Br)ccc2C(F)(F)C[NH+]2CCCC2)cc1', 'C[C@@H](Nc1cccc(Cl)c1)[C@@H]1CCC[NH2+]C1', 'Oc1cc(O[C@@H](CN2CCOCC2)[C@@H]2CCC[NH+]2Cc2ccccc2)ccc1O', 'O=[N+]([O-])c1ccncc1[C@@H]1CSC(F)(F)C1=O', 'O=[N+]([O-])c1cccc2nnc(C[NH+]3CCC[C@@H](CCc4cccnc4)C3)nc12', 'Clc1ccc(C[NH2+][C@@H]2CCC[C@H](c3nc(C(F)(F)F)cs3)C2)cc1', 'Clc1cc(Br)c2nc([C@H]3[C@@H]4CCCC[C@H]4C3)c[nH]c12', 'O=C(/C=C\\C(Cl)(Cl)Cl)[C@H]1CCS(=O)(=O)C1', 'Clc1ccccc1O/Cn1c2ccccc2[nH+]2ncnc21', 'CN(CC1CC[NH+](Cc2ccon2)CC1)C(=O)c1c[nH]cn1', 'Cc1nccn1-c1cccc(C[C@H]2CCC[NH2+][C@H]2C)c1', 'Clc1nncc(Cl)c1N1CCN(C(=O)[C@@H]2C=c3ccccc3=[NH+]2)CC1', 'Clc1ccc([C@@H]([NH2+][C@H]2CCc3ccc(F)cc32)[S@@](=O)Cc2ccccc2)nc1', 'Oc1cc(F)c(-c2ccco2)cc1N1CCC[C@@H]1C[NH+]1CCCC1', 'Clc1ccc(Cc2c[nH]nc2C[NH+]2CCCC[C@@H]2c2cccnc2)cc1', 'O=C([O-])[C@@H]1CO[C@H](c2ccccc2)S1', 'O=C(c1ccsc1)[C@@H]1SC(N2CC2)c2ccccc2S1(=O)=O', 'Cc1snc(C[NH+]2CCC(C(=O)NCc3scnc3C)CC2)n1', 'C[C@@H]([NH2+]C[C@H](O)C1CCOCC1)c1cccc(OCC#N)c1', 'OC[C@@]1([NH2+]Cc2cccc(Br)c2)CCCC1(C)C', 'NS(=O)(=O)Cc1cn2ccccc2[nH+]1', 'O[C@@H]1C(/C=N/O)C=CC(=O)C1(CO)CCc1ccccc1', 'Clc1ccc([C@@H](N[C@@H](CO)c2ccc(Cl)cc2)[NH+]2CCCC2)c(F)c1', 'O=[N+]([O-])c1cc([C@H]2[C@@H]3OCC[C@H]3CC[C@H]2O)cc([N+](=O)[O-])c1O', 'O=C(c1cc(Br)c[nH]c1=O)[C@@H]1CCC[NH+](Cc2ccc(Cl)cc2)C1', 'Clc1cc(F)c(NC2CC[NH+](CCCC(=O)[O-])CC2)cc1', 'OCc1ncsc1OC[C@H]1CN(Cc2ccccc2)C2(CCOCC2)C1', 'O=[N+]([O-])c1ccn(CCC[NH+]2CCCC[C@@H]2C2CCCCC2)c1', 'Clc1ccc([C@@H]([O-])[C@H]([NH2+]CC(=O)NC2CC2)C2CC2)cc1', 'Clc1cccc(Br)c1C[NH+]1CCCC[C@@H]1CCl', 'Clc1nnns1\n\n\n\n\n\n\n\n', 'Clc1nnns1\n\n\n\n\n\n\n', 'Clc1nnns1\n\n\n\n\n\n', 'Clc1nnns1\n\n\n\n\n', 'Clc1nnns1\n\n\n\n', 'Clc1nnns1\n\n\n', 'Clc1nnns1\n\n', 'Clc1nnns1\n', 'Clc1nnns1', 'Clc1nnns1', 'Clc1nnns1', 'Clc1ccccc1-c1csc(CC[NH2+]C2CCC3(CC3)C2)n1', 'OCc1nc[nH]n1', 'OCc1nc[nH]n1', 'Clc1ccc(Br)cc1C1CC[NH+](C[C@@H](O)C2CCOCC2)CC1', 'O=C(c1ccsc1)[C@@H]1[C@@H]2C[C@@H]2CC[C@@H]1C[C@@H]1CCCO1', 'Clc1cc(F)c([C@H](Oc2ccccc2)[C@@]2(C(F)(F)F)NC(=S)N=C2C(=O)[O-])cc1', 'Oc1cccc2cc(O[C@H]3CCC[NH2+]C3)ncc12', 'O=C(CSC(=S)[C@@H]1CCCC[NH+]1C1CC1)Nc1cncc(-c2ccccc2)n1', 'Clc1cccc2c1CCC[C@H]2[NH2+][C@@H](c1cccc(CC(F)F)c1)[C@H]1CCS(=O)(=O)C1', 'C=C([O-])c1ccc(N2CC(=O)NC3=C2CC[C@H]3C)cc1Cl', 'ClCc1n[nH]c(S[C@@H]2CCCC[C@H]2Cn2ncn(CC3CC3)c2=O)n1', 'Clc1nnnn1/N=C/[C@@H]1CCCC[C@H]1O', 'Clc1cccc(NC[C@@H]2CC[NH+](CN3CCC(O)CC3)C2)c1', 'N=CCCOc1ccc(O[C@H]2N=NC(=O)[C@@H]2C)cc1Cl', 'Clc1ccsc1/C=C/CC[NH2+]C(CC(=O)[O-])c1ccccc1', 'Clc1ccccc1N1Nc2[nH+]ccnc2NC1', 'Nc1c(Oc2[nH+]cccc2F)[nH]c2c1-c1ccc(Br)cc1NC2', 'Oc1cccc([C@H]2O[C@H](Br)[C@H](F)[C@H](O)C2)c1', 'O=C([O-])c1c(N2CC(c3ccccc3)n3nc(C4CCCCC4)ncn23)coc1-c1noc2c1CCCC2', 'ClCc1ccn([C@H]2CCO[C@@H]2C(F)(F)F)c1', 'NS(=O)(=O)CCC[NH+](C(=O)c1ccc(Cl)s1)c1ccccc1', 'Oc1ccc(O[C@@H]2CCC[NH2+]C2)cc1Cl', 'Clc1cccc([S@@]c2c[nH]nc2C[NH+](CC2CCCC2)C2CC2)c1', 'Clc1ccc([C@@H]([C@@H]2C[NH2+]CCO2)N2CCOCC2)cc1', 'O=[N+]([O-])c1c(Br)sc2c1C(=O)[C@@H](c1ccccc1)[C@@H]1CC=C[C@H]21', 'Oc1cc(F)ccc1N[C@@H]1CCO[C@]2(CCSC2)C1', 'ClC[C@@H]1C[C@H](O)C[NH+]1Cc1nc2ccccc2c2ccccc12', 'Clc1ccc([C@@H](C)[NH2+][C@H]2CCCNC2=O)cc1', 'CNC(=O)c1cc(CN2CC[NH+](C)C[C@H]2c2cccc(Cl)c2)no1', 'Clc1cccc([C@@H]2[C@@H](c3ccc(Cc4cnn[nH]4)nc3)N=C2Cl)c1', 'Clc1ccc(CNSc2ncnc3c2C[NH+](CC(F)(F)F)C3)cc1', 'O[C@@H](C[NH+]1CCC[C@H]1c1ccccc1)C1CC1', 'Clc1ccccc1[C@H]1[C@H]2NC(=S)N=C2S1', 'Clc1cccc(SC[C@H]2CCCC[C@@H]2[NH2+][C@@H]2CCOC2)c1', 'Clc1cc(Cl)nnc1NCc1cnn([C@@H]2CCC[NH2+]C2)c1C1CC1', 'Oc1cccc(C2=NN([C@H]3[C@@H](C(=O)[O-])C[C@H](O)N(c4ccccc4)[C@H]3c3ccccc3)C2)c1', 'Clc1ccc(C[NH+](C[C@@H]2CCOC2)C[C@H]2CCCO2)cc1', 'Clc1cccc2[nH]nc(S(=O)(=O)N3CCCC[C@@H]3C[NH+]3CCCC3)c12', 'Oc1ccc(Cn2cc([C@@H]3CC[NH2+]C3)nn2)cc1I', 'Clc1ccsc1-c1n[n-]c(C(=O)N2CCCC[C@H]2c2ncccn2)n1', 'CC[S@](=O)[C@H]1CCC[C@@H](NC(=O)N(CCC(F)(F)F)C2CCCC2)C1', 'O=C([O-])[C@@H]1Cc2cc(-c3cccs3)[nH]c2N1', 'Clc1ccc(Br)cc1S(=O)(Br)F', 'Clc1cc(F)c(F)cc1C[C@H]1CCO[C@@]2(CCOC2)C1', 'Clc1ccsc1SC1CCN(c2ncnc3[nH]c[nH+]c23)CC1', 'O=C(c1cn[nH]n1)N([N+]1CCC1)[C@@H]1CCSc2ccccc21', 'Clc1ccc(Br)cc1-c1cnn([C@@H]2CCC[NH2+]C2)n1', 'Clc1cccc2[nH]nc(S[C@H]3CC[NH2+]C[C@@H]3Cc3ccccc3)c12', 'N=Cc1nonc1N', 'COc1nc2snnc12', 'Clc1cccc2nn(S[C@@H](NC3=CCCC3)C(F)(F)F)nc12', 'O=[N+]([O-])c1c([O-])[nH]c(=O)[nH]1', 'Clc1ccc(C(N)=N\\O[C@H]2CCO[C@]3(CCOC3)C2)cc1', 'N#Cc1ccccc1O[C@@H](C)C(=O)N1CCOC[C@@H]1C1([NH+](C)C)CCCCC1', 'Clc1ccc(Br)cc1[C@@H]1C[NH+](C2CC2)CN1Cc1cccc(F)c1', 'Clc1ccc(Br)cc1C[C@H]1CCC[NH+](CC2CC2)C1', 'Clc1cccc(NCC[NH+]2CCC[C@@H]2CN2CCOCC2)c1', 'Oc1ccc(F)cc1[C@H]1[C@@H]2C(=O)N(c3cccnc3)[C@@H]2[C@@H]2C=CC[C@H]21', 'O=[N+]([O-])c1ccncc1COC[C@H]1CCCC[C@@H]1n1cc[nH+]c1', 'ClC[C@H]1O[C@H]2[C@@H](c3ccccc3)N=C12', 'Cc1noc(C)c1S(=O)(=O)/N=C(\\[O-])[C@]1(C)CC1(Cl)Cl', 'Clc1ccc(Br)c(Cl)c1C[NH+]1CCC(O)CC1', 'Clc1ccc(Br)cc1O[C@@H](CCC12CC=CCC2)C1', 'O=[N+]([O-])c1ccn([C@H]2NC[C@@H](O)[C@@H]2O)c1', 'Oc1ccc(O[C@H]2CS(=O)(=O)C[C@@H]2C[NH2+]C2CC2)cc1F', 'O=[N+]([O-])c1ccncc1/C=N/N=C[C@H]1C=C[C@@H](O)C1', 'N#C[C@@]1([NH2+]C2CC2)CCC[C@@H]1CCSc1ccccc1', 'Clc1ccc(Br)c(S[C@H]2CCO[C@]3(CCSC3)C2)c1', 'O=[N+]([O-])c1c(Cl)nc2ncon12', 'O=C(/N=C1\\CC[NH+]=C1C1OCCO1)c1cnc(Cl)cc1Br', 'OC[C@@]1(N2CCc3ccccc3C2)CCC[NH+]1C1CCCCC1', 'Oc1ccc(F)c([C@@H]2CCC[NH+]2Cc2ccccc2CO)c1', 'Clc1ccc(Br)cc1S(=O)([O-])Br', 'Clc1cccc([C@@H](N[C@H]2CC[NH+](Cc3ccco3)c3ccccc3)C2)c1', 'Oc1cc(O)c(N[C@@H]2CC[C@H]([NH3+])C2)cc1Cl', 'Clc1ccc(Br)cc1/C=N/N=C1\\S[C@@H]2CS(=O)(=O)C[C@H]12', 'O=C(/C=C1CCOC[C@@H]1N1C[C@H]2CCCC[C@H]12)N1CCOCC1', 'O=C([C@H](c1ccc(Cl)cc1Cl)N1CC[NH+](C2CCCCC2)C1)N1CCOCC1', 'Clc1cccc2nc(C[NH+](Cc3ccco3)[C@@H]3CCS(=O)(=O)C3)ccc12', 'Clc1cccc(F)c1O[C@@H]1CCO[C@]2(CCSC2)C1', 'Clc1ccccc1OCc1csc(C(=O)N[C@@H](CCS2CCOC2)C(F)(F)F)c1', 'Clc1ccc(Br)c(F)c1NC1[C@@H]2CC[C@@H]1C2', 'Oc1ccc(Oc2cc([C@@H](O)C[NH+]3CCCC3)ncn2)cc1F', 'Clc1nnc(N[C@@H]2C[C@H]2[N+]2=NCCS2)nc1NCc1ccccc1', 'CC1=NO[C@@H](C(=O)N2CCC[C@H]2C(=O)[O-])C1', 'Clc1ccccc1[C@H](O)CCCNC[C@H]1CCCC[NH+]1C', 'Clc1ccc(Br)c([C@@H]2[C@H](C(=O)[O-])CCO2)n1', 'Clc1cc(C2CC[NH+](C[C@H](c3cccnc3)c3cccs3)CC2)ncn1', 'O=[N+]([O-])c1ccc[nH+]c1SCCF', 'Clc1ccsc1[C@@H](F)C(=O)[O-]', 'ClCc1cnn[nH]1', 'Oc1cc(F)ncc1S[C@@H](CO)Cc1c[nH]nc1C1CC1', 'Clc1nncnc1C[C@H]1CCC[NH+](CCc2c[nH]c3ccccc23)C1', 'Clc1cc(NC[C@@H](n2cncn2)[NH+]2CCCC2)ccc1F', 'Clc1ccc([C@@H]([O-])C[NH+](Cc2ccco2)C2CC2)cc1', 'O=C([O-])[C@@H]1C[C@](O)([C@H](F)C(F)(F)F)N(Cc2ccc(Cl)cc2)C1', 'NS(=O)(=O)CC[NH+]1CCc2ccccc2C1', 'Clc1nncnc1[N-]Sc1ccncc1', 'Oc1cccc([C@H](OCC[NH3+])C(=O)[O-])c1', 'Oc1cc(-n2cc[nH+]cc2=O)cc(I)c1', 'Oc1cc(C[NH+]2CCCC2)cc([N+](=O)[O-])c1O', 'Oc1cc(F)nc(S[C@@H]2CCOC3(CCOCC3)C2)c1', 'Clc1cccc2ccc(SCCC[NH+]3CCC[C@@H](n4cccn4)C3)nc12', 'NS(=O)(=O)CCCC1CC[NH+](CCCC2=CCCCC2)CC1', 'Clc1ccsc1[C@H]1N=C(C[NH+]2CCN(c3ccc(Cl)cc3)CC2)C1', 'CC[C@@H]1C[NH+](Cc2ccccc2)C[C@@H](C)C1', 'Clc1ccc(-c2nn([C@@H](c3ccccc3)[NH+]3CCC(OC(F)F)C3)cc2F)cn1', 'ClCc1cn(C[NH+]2CCCC[C@H]2c2ccccc2)nn1', 'Oc1ccccc1-c1noc([C@@H]2CCO[C@]3(CCSC3)C2)n1', 'Clc1nncc(N2CCSC[C@H]2Br)n1', 'Clc1cc(N2CCC(C[NH+]3CCCC3)CC2)nc([O-])c1', 'O[C@@H](c1csc(C2CCCCC2)n1)[C@H]1CCOC(O)(CO)C1', 'Clc1ccccc1O/N=[N+]=SP1CC1', 'ClC[C@H](O)[C@@]1(Cc2ccccn2)N2Cc3ccccc3[C@H](O)[C@H]12', 'COc1nc(C(=O)N2CC[C@]3(CCCC[C@H]3C)C2)no1', 'ClCc1nc[nH]n1', 'O=C([C@H](CNc1ncccn1)[NH+]1CCCC1)NN[C@@H]1CCS(=O)(=O)C1', 'Oc1cc(O[C@@H]2CCO[C@@]3(CCOC3)C2)c(Cl)cc1Cl', 'O=[N+]([O-])c1c([O-])sc2scnc12', 'CCCC1CC[NH+]([C@H](C)c2nnnn2CC)CC1', 'O=[N+]([O-])c1c(F)cccc1NC[C@@H]1CCC[NH2+][C@@H]1CO', 'Oc1ccc(O[C@H]2CCOC3(CCOC3)C2)cn1', 'O=C(/C=C\\C1(C(=O)Nc2cncc(-c3ccccc3)n2)CC1)N(C1CCCC1)S(C)c1cccnc1', 'Oc1cc(C[NH2+][C@@H]2CCCC[C@H]2CCO)cc(Cl)c1O', 'O=C(C1=Nc2sccc2[C@@H]1O)[C@H]1CSC(=S)N1c1ccccc1', 'O=C(Cc1conc1C1CC1)NC[C@@H]1CC[C@@H]([NH2+]C(C)C)C1', 'Clc1cc(/N=C\\C=C/C[NH+]2CCCC2)ccc1Cl', 'O=[N+]([O-])c1ccn(C2CC[NH2+]CC2)c1', 'Oc1ccc(O[C@H]2C=[NH+]/C(=C/c3cccnc3)SC2=S)cc1', 'Clc1ccc(-c2ccccc2[N-]C[C@H](O)C[NH+]2CCCC2)cc1', 'Oc1cccc(CCn2ccnc2NC[C@@H](O)C[NH+]2CCCC2)c1', 'Nc1[nH]ncc1C[NH3+]', 'Clc1ccccc1Oc1cc(C[NH2+][C@@H]2CCOC3(CCCCC3)C2)ncc1SC', 'O[C@@H]1C(=O)N(Cc2cnc3ccccc3c2[O-])C=C[C@@H]1O', 'ClC[C@@H]1OCC[C@@H]1CN[C@H]1CCCC[C@@H]1CO', 'N=Cc1n[nH]c(C(=O)N2CC[C@@H](C)[C@@H](O)C2)n1', 'Clc1cccc(S[C@@H]2C[NH2+]C[C@H]2CO)c1', 'OC[C@H]1CC[C@]2(CCc3c(F)cc(Br)cc3O2)C1', 'ClC[C@H]1C[NH+](Cc2ccccc2)CC1', 'Clc1ccccc1[C@H]([NH3+])C[NH2+]C[C@@H]1CCCO1', 'N=C1C[NH+](C2CC[NH+](CC(N)=O)CC2)CCN1Cc1ccccc1', 'Cc1ccsc1[C@H]1[NH+]=c2ccccc2=C1N', 'ClC[C@@H]1Oc2ccccc2C[NH+]1CCC1CC1', 'Oc1cccc2cc(O[C@H](Cc3ccccn3)[NH+]3CCCC3)nnc12', 'CN=C(C#N)CCC(=O)N1C[C@H](C)OCC1(C)C', 'OC[C@@]1([C@H]O)NCc2ccc(Br)cc21', 'Nc1ncccc1[S@@](=O)Cc1nc[nH]n1', 'C=CCC[C@H]1CCC[NH+](Cc2cccc(NC(C)=O)c2)C1', 'Clc1cc(/C=N\\NC2=N[C@H]3OCO[C@@H]32)ccc1Br', 'O=[N+]([O-])c1ccnc2c1C[NH+](Cc1ccc(C(F)(F)F)c(Cl)c1)CC2', 'O=C([C@H](Cl)c1nc2ccccc2s1)[NH+]1CCCCC1', 'O=[N+]([O-])c1ccc(NC[C@@H]2C[C@@H](O)C[NH+]2Cc2ccccc2)cc1F', 'C=C[C@@H]1N[C@]2(C(=O)NCCN2c2ccccc2)c2ccccc21', 'Clc1ccc(CO)c([C@@H]2CSCC[NH2+]2)c1', 'Clc1cc(CNC[C@H]2C[NH2+]CCO2)ncn1', 'Clc1ccc(Br)cc1C[NH2+][C@@H]1CCC[C@H]1O', 'O=[N+]([O-])c1ccc[nH+]c1I', 'Oc1ccc(/C=N/N[C@@H]2CCSC2)[nH]1', 'Clc1ccc(Br)cc1O[C@H]1CC[C@@H]([NH+]2CCCC2)C1', 'Clc1ccc(Br)cc1O[C@@H]1CC[C@@H]([NH+]2CCCC2)C1', 'Oc1cccc2ncc(C[NH2+][C@@H]3CCSc4ccccc43)n12', 'Clc1cccc(S(=O)(=[NH2+])CC2C(C)(C)CS2)c1', 'O=[N+]([O-])c1c(-c2cc(Cl)ccc2Cl)[nH]c2c1C[NH+](C[C@@H]1CCCCO1)CC2', 'OCS[C@@H]([NH2+]C[C@@H]1CCCO1)c1nccn1C1CCCC1', 'Clc1ccc(C[NH+]2CCC[C@H](c3nc(-c4cccnc4)no3)C2)s1', 'C=CC[NH+](CC(=O)[O-])Cc1ccc(OC)c(OC)c1', 'Clc1ccsc1-c1ccoc1C[NH+]1CCCC1', 'ClCc1nnc([C@]2(CC(F)(F)F)N=CN2C(=O)c2ccsc2)s1', 'O=[N+]([O-])c1cc([C@H]2[C@H]3CCCC(=O)[C@H]3[C@H]3CCCC[C@H]3O2)ccc1C(F)(F)F', 'Clc1ccccc1[C@H](O)[C@@H]1CCO[C@]2(CCSC2)C1', 'O[C@@H](C(=O)NC[C@@H]1CCC[C@H](S(=O)(=O)N2CCCC2)C1)[C@H]1CCS(=O)(=O)C1', 'OC[C@@]1(O)CCSC1', 'CC[C@@H](C(=O)NCCC1=CCCCC1)C(C)(C)C[NH+](C)C', 'Oc1ccc([C@H]2OC[C@@H](O)C[C@@H]2C[NH2+]Cc2cccc(Br)c2)cc1', 'Clc1nncc(N[NH+]2CCC[C@@H](C(=O)N3CCCCC3)C2)c1Cl', 'Oc1ccc(F)c(/C=C2\\N=c3ccccc3=[NH+][C@@H]2c2cccc([N+](=O)[O-])c2C(F)(F)F)c1', 'O[C@@H](c1cc(OC(F)F)cc(Br)c1)[C@@]1([NH+]2CCCCC2)CCOCC1', 'Clc1ccc(Br)cc1[C@@H]1C[NH2+][C@@H](CO)[C@H]2NC(=S)N(Cc3ccccc3)C12', 'ClC[C@@H]1O[C@H]2OCCN2[C@@H](c2cccnc2)O1', 'O[C@@H](Cn1cccn1)[C@H]1CSCCS1', 'O=[N+]([O-])c1cc(N2CC[NH+](CC3=c4ccccc4=[NH+]C3)CC2)ncc1Cl', 'Nc1ncc2c(c1O)C[NH+](Cc1cccc(C(F)(F)F)c1)CC2', 'NS(=O)(=O)N[C@@H](c1ccccc1Cl)C1([NH+]2CCOCC2)CCCC1', 'Clc1ccccc1[C@H]1[C@@H]([O-])CC(O)=N[C@@H]1N1CCOCC1', 'COC(=O)[C@H]1CCCC(C)(C)[C@@H]1[NH3+]', 'O[C@@H]1[C@@H](C2CC([O-])O[C@@H]2c2ccc(Cl)cc2)CC1', 'ClCc1ccc2nsnc2c1C[NH+]1CCCC1', 'O=C(CS(=O)(=O)[N-]c1ccc(OC(=O)c2cc(F)ccc2F)cc1)N1CC[C@H]([NH+]2CCCC2)C1', 'Clc1cc(Cl)c(C[NH+]2CCC[C@H]2CN2CCOCC2)cc1Cl', 'ClC[C@H](O)[C@H]1CN2CCC[C@@H]2CO1', 'CCC[C@H]1CCC[C@@H]1C(=O)NCCCC[NH+](C)C', 'Oc1cccc([C@H](c2[nH]nc3ncccc23)[NH+]2CCC(O)CC2)c1', 'N[C@@H]1[NH2+]CC(C)(C)[C@H]1C[C@@H](CO)c1ccccc1', 'Oc1cccc(Br)c1C[NH+]1CCC[C@@H]1c1cncn1C1CC1', 'Clc1nnc(N[C@H]2CCOC3(CCOCC3)O2)cn1', 'Cc1ncsc1C[NH+]1CC[C@H](Oc2ccccc2)C1', 'C[C@@H]([C@@H](C)O)[NH+]1CCN(C(=O)Cc2csc(C(C)C)n2)CC1', 'Clc1ccccc1[C@H]([NH2+]CCO[C@H]1CCS(=O)(=O)C1)c1nccs1', 'O=[N+]([O-])c1cc(Cl)c(CN2CCO[C@@H](CO)[C@@H]2C[NH+]2CCCC2)cc1', 'N#C[C@@]1([C@H](O)c2ccco2)CCC1(C)C', 'OC[C@@H](CNc1ccnc(Cl)c1)N[C@@H]1CC(CO)(c2ccccc2)n2ccccc2=N1', 'Clc1ccsc1[C@@H]1Oc2c(nnc3c2SCC3)C1', 'OCSCCCN1CC[NH+](C2CCCCC2)CC1', 'Oc1ccccc1[C@H]1CC[NH2+][C@@H]2[C@H]1c1ccc(F)cc12', 'Clc1ccc(Br)c(C[C@@H]2CCC[NH2+]2)c1', 'Clc1ccsc1C[N-]F', 'N=CCOC(=O)C1=c2ccccc2=[NH+][C@H]1Cn1ccnc1', 'Oc1cc(Br)c([C@H]2NN3[C@H](CC2)OC3C(=O)C2=CCCCC2)cc1Cl', 'Clc1ccsc1S(=O)(=O)C1CC[NH2+]CC1', 'CNC(=O)[C@@H]1C[C@@H](O)C[NH+]1CC(=O)Nc1ccccc1F', 'O[C@@H]1C[C@H](c2cccs2)C2(CCCCC2)N1[C@@H](Cn1cccn1)N1CCOCC1', 'Oc1ccc(O[C@H]2CC[NH2+]C[C@@H]2CC2CC2)c(Cl)c1', 'Clc1nnn(-c2cncc(C[NH+]3CCOCC3)c2)c1Cl', 'NS(=O)(=O)/N=C(/[O-])c1ccccc1C1([NH+]2CCCCC2)CCSCC1', 'ClC(/N=C1/N=C[C@@H]2c3ccccc3C(=O)N12)[C@@H]1COc2ccccc21', 'Cc1ncc([C@H](C)[NH2+]C(C)(C)C)s1', 'ClC(=S)N(Cc1ccccn1)[C@H]1C[C@@H]2CC[C@@H]1C2', 'OC[C@@H]([NH2+][C@@H]1CCOC2(CCCC2)C1)C(=O)NCc1ccccc1', 'Clc1cc(Br)cnc1O[C@@H]1CC[C@@H]([NH2+]C2CC2)C1', 'OC1=C[C@H](Sc2c[nH]c3cccc([N+](=O)[O-])c23)C([O-])=N1', 'COC[C@@H]([NH2+]CC1CCCC1)C1CC1', 'O=C(/C=C1/c2ccccc2S1)[C@]1(Nc2ccccc2)SC2=c3ccccc3=[NH+][C@@H]21', 'ClC[C@@H]1CCC[NH+]1Cc1ccccc1O', 'Clc1ccc(Br)c(F)c1C[NH+]1CCC[C@H](c2nncn2C2CC2)C1', 'Clc1ccsc1[C@@H]([NH2+][C@@H]1CCCc2cccnc21)C1CC1', 'Clc1cc(Br)c2nc([C@H]3CCC[NH2+]C3)sc2c1', 'NS(=O)(=O)N[C@H]1CC[NH+](Cc2ccc(F)cc2)C1', 'OC[C@@H](O)C[NH+]1CCC([C@H](O)c2ccc(Br)cc2)CC1', 'ClC[C@@H]1C(C#N)C=CC[C@@]1(O)c1ccccc1', 'O=C(c1cnc([C@H]2CCC[NH+](C/C=C\\c3cccs3)C2)o1)N1CCCC1', 'Clc1nnnn1/C(CCC(F)(F)F)N[C@H]1CC[C@@H](SCC(F)(F)F)C1', 'Oc1cc(-n2nc([C@@H](Cl)C(=O)[O-])ccc2=O)co1', 'Clc1ccc(Br)cc1[C@@H]1O[C@@H]2CCC[C@H]2[C@@H]1[NH3+]', 'Clc1ccc(-c2csc3scnc23)c[nH+]1', 'O=C(C1CC1)C1=CC[NH+]([C@H]2CCN(c3cc(F)ccc3F)C2)CC1', 'O=[N+]([O-])c1ccc[nH+]c1[C@@H](N1CCOCC1)[C@@H]1CCS(=O)(=O)C1', 'O=C(Cc1nc2ccccc2n1Cc1ccccc1)Nc1ccc2c(c1)OCCCO2', 'Oc1ccc(F)cc1[C@H](CO)NC1CC[NH+](C2CC2)CC1', 'Oc1ccc(/N=c2\\sc(Cl)cc(Cl)c2C[NH+](CC(=O)[O-])c2ccccc2)cc1', 'Clc1cccc(C[NH2+][C@H]2CC[C@@H](S)C2)c1', 'O=[N+]([O-])c1ccn(CC[NH+]2CCC[C@@H]2c2ccc(O)cc2)c1', 'Clc1ccc(-c2nn([C@@H]2CC23CC[NH+](Cc4ccccc4F)CC3)C2)cc1F', 'O=C([O-])c1[nH]ncc1CNC(=O)C[C@@H]1CCCC[NH2+]1', 'O[C@@H]([C@@H]1CCOc2ccccc21)[C@@H]1CCOC2(CCOCC2)C1', 'Oc1cc(Br)cnc1S[C@@H]1C[C@@H](Br)CO1', 'O=[N+]([O-])c1cc(O[C@@H]2CCCC[C@H]2[C@@H]2CCC[NH2+]C2)c(Cl)cc1Cl', 'O=C(Cc1conn1)NCCN1CCSC[C@@H]1C[NH+](C)C', 'ClC(=S)[C@@H]1CCCS1', 'Clc1cc(Br)ccc1C[NH+]1CCC[C@@H](CCO)C1', 'C=C1NN(C(=O)c2cnccc2)[C@H]2N=CN=C2[C@H]1C', 'CN[C@H]1CC[C@@H]([NH+]2CCC(Cc3nc(C4CC4)no3)CC2)C1', 'Clc1cc(F)cc(C[NH+]2CCC[C@H](c3nc(-c4ccco4)[nH]n3)C2)c1', 'Clc1ccc(Br)cc1[C@@H]1C[C@@H](CI)c2[nH]cnc2S1', 'Clc1cc(Br)c(S(=O)(=O)[C@@H]2CC[NH2+]C2)cc1F', 'Clc1ccc(C[NH+](CCCn2cccc2)C2CC2)s1', 'CCn1nc2n(c1=O)CCC[C@@H]2[NH2+][C@H]1c2ccccc2C[C@H]1OC', 'O=C(C[NH+]1CCN(C(=O)CCC2C[C@@H]3CCCC[C@H]3C2)CC1)NC1CCCCC1', 'N=CC(=O)NC1CC[NH+](CCCn2ncc(C(F)F)n2)CC1', 'O=[N+]([O-])c1ccn([C@H](OC[C@H](O)c2cccc(Cl)c2)[NH+]2CCCC2)c1', 'O=C(CNC(=O)C[NH2+]C1CC1)N1CC[C@](O)(CO)C1', 'O=C(C[NH+]1CCC2(CC1)NCCN2C(=O)c1ccc2c(c1)OCC(=O)N2)OCCO', 'Clc1cc(/N=C\\C=C/CC(=O)[O-])c(Br)s1', 'N[C@@H]1[NH2+]CC[NH+]([C@H](Cc2ccccc2)c2ccccc2)C1', 'Oc1ccc(F)c(C[C@@H]2CCC[NH2+]2)c1', 'Nc1[nH]nc(CN[C@@H]2CC[C@@H]([NH+](Cc3ccccc3)C(=O)NCCCC(=O)[O-])c2)c1Cl', 'O=[N+]([O-])c1cc(O[C@@H]2CCCC[C@@H]2CCC[NH+]2CCCC2)c(NC2CC2)cc1', 'Clc1nnnn1c1[nH]cnc1C[C@@H]2CCC[NH+](Cc1ccccc1Cl)C2', 'CCS(=[NH2+])=c1/c(=N/C(=O)COC)sc2c(C)cc(C)cc12', 'Clc1ccc([C@@H](Br)C[NH+]2CCC(C(N)=O)CC2)s1', 'ClC(F)(CBr)c1c[nH]cn1', 'Clc1ccc(CC[C@@H]2CCC[NH+](CC3CC3)C2)c(O)c1', 'O=C(CC[C@@H]1NC(=O)c2ccccc2N1CC1[NH+]=c2ccccc2=[NH+]C1=O)N[C@H]1CCCC[C@@H]1O', 'Clc1ccsc1[C@H](Cl)[C@@H]1CCOC2(CCCC2)C1', 'O=[N+]([O-])c1ccncc1C[NH+]1CCN[C@H](c2ccc(Cl)c(Cl)c2)C1', 'O=[N+]([O-])c1c(OCO)sc2cc(O[C@H](CSc3ccccn3)[NH+]3CCCCC3)ccc12', 'Clc1nnnc(C[NH3+])c1Cl', 'ClC[C@H](C[NH+]1CCc2cc(Cl)ccc2N1C[C@@H]1CCCO1)N1CCCC1', 'Oc1ccc([C@H](CNC(=O)c2cnc([C@H]3CCCCO3)n2C2CC2)[NH+]2CCCC2)s1', 'O=[N+]([O-])c1ccn(CC[C@@H]2CCCC[NH2+]2)c1', 'NS(=O)(=O)Nc1cc([C@@H]2CCC[NH+]2CC2CC2)ccc1Cl', 'N#C[C@@H]1C(=O)NCCN1CC1CC[NH+](Cc2ccccc2)CC1', 'ClC(/C=N/O)c1c[nH+]ccc1N1CCOCC1', 'Oc1ccc(F)cc1S[C@@H]1CC[C@@H]([NH+]2c3ccccc32)C1', 'CN(c1cc(Cl)cc(N)n1)C1CC[NH+]([C@@H](C)C(C)C)CC1', 'O=C(c1cc(F)c2[nH]ccc2n1)N1C[C@@H]2CC[C@@H](C1)N2C(=O)Nc1cccnc1Cl', 'O[C@@H]([C@@H]1C=c2ccccc2=[NH+]1)c1cc([N+](=O)[O-])ccn1', 'ClC[C@@H]1CC[C@@H](N[C@@H]2CCCC[C@H]2O)[C@H]1CO', 'N#CCc1ccc(OCC[NH+]2CCC(CN3C[C@H](C)O[C@H](C4CC4)C3)CC2)cc1', 'ClC[C@@H]1CCC[NH+](Cc2csc(-c3cccnc3)n2)C1', 'O=C(CS(=O)(=O)[C@@H](C(=O)[O-])C(F)F)N[C@@H]1CCCC[C@@H]1O', 'O=C(c1cc([S@](=O)CC(=O)Oc2ccccc2F)cs1)[NH+]1CCCC1', 'Clc1ccc(Br)c(/C([O-])=N/N2CC[NH2+]CC2)c1', 'O=[N+]([O-])c1ccn([C@H]2CCCCN2C2CC[NH2+]CC2)n1', 'Oc1ccc(F)cc1C[NH+]1CCN(c2cc[nH+]cc2Cl)CC1', 'O[C@@H]([C@@H](SCC(F)(F)F)C(F)(F)F)NC[C@H]1CCCO1', 'O=C(C[NH+]1CCO[C@]2(CCCN(c3ncccn3)C2)C1)NC1CCCCC1', 'N=CC[NH+]1CCCC[C@@H]1c1ccc(S(=O)(=O)C(F)F)cc1', 'ClC(c1ccccc1Cl)[NH+]1CCC[C@H](n2cccn2)C1', 'N#C[C@@H]1C[NH+](Cc2ccc(F)cc2)C=C1c1ccc(Br)cc1O', 'Clc1cc(C[NH+](Cc2ccco2)Cc2ccc(Cl)o2)co1', 'Clc1ccc([C@@H](O)[C@@H]2[C@@H]3CC[C@H](C3)[C@@H]2O)cc1', 'Clc1ccccc1-c1noc(C[NH+]2[C@@H](C(=O)[O-])CCCN2)n1', 'O=C(c1ccsc1)[C@@H]1CS[C@@H]2[C@H](C=C3C=CC=CC3=N2)C1', 'Clc1ccc([C@@H](CO)[NH2+]C2CC2)s1', 'Oc1ccccc1C[C@H]1CCC[NH+](C2C[NH+](c3nc(OCc4ccccc4)cs3)C2)C1', 'Oc1cc(F)c([O-])c(C(=O)O[C@@H]2CCCCC2(Cl)F)c1Cl', 'Clc1nnc(SCC2CC[NH2+]CC2)nn1', 'ClCc1ncoc1[C@@H]1CCC[NH+](Cc2ccc(F)cc2)C1', 'Clc1cc(F)cc(N2CC[C@@H]([NH2+][C@H]3CCNC3=O)C2)c1', 'O=[N+]([O-])c1c([N+](=O)[O-])cncc1C[C@@H]1CCC[NH+](C2CCCC2)C1', 'O=C(C1=c2ccccc2=[NH+][C@@H]1C[NH+](C)C(C)C)[C@@H]1CC(=O)N(c2ccccc2F)C1', 'Clc1cc(F)c([O-])c(C[NH+]2C[C@@H](CO)C2)c1', 'O=C(CS(=O)(=O)[C@H]1CCCC[C@@H]1[NH2+]C1CC1)N1CCC(CC(N)=O)CC1', 'Clc1cc(F)cc(-c2ccncc2)c1C[NH2+][C@H]1CC[NH+](Cc2ccccc2)C1', 'Clc1ccsc1C[N-][C@H]1[C@H]2CCO[C@@H]2c2ccccc21', 'Oc1ccccc1[C@@H]1C[C@H](O)C[NH+]1Cc1ccc([N+](=O)[O-])cc1C(F)(F)F', 'Clc1cccc(F)c1-c1[nH+]ccn1CCC[NH+]1CCCC1', 'Clc1nnnc(S[C@H]2CCO[C@@]3(CCC[C@@H]3c3ccccc3)C2)n1', 'O=C(c1cccc(-c2noc(-c3cc(F)ccc3F)n2)c1)[NH+]1CC[C@H]([NH+]2CCCC2)C1', 'Oc1ccccc1[C@@H]1[NH+]=c2ccccc2=[NH+][C@@H]1c1cccc(Cl)c1', 'N[C@@H](c1c(F)cccc1F)[C@H]1CCO[C@]2(CCSC2)C1', 'N#CCS[C@@H]1CCC[NH+](C2CCN(c3cc(Cl)ccc3)CC2)C1', 'NS(=O)(=O)C[C@H](Br)C(=O)[O-]', 'Clc1ccc(C[NH+](CCO)[C@@H]2CCC[C@H]2S(C)(=O)=O)cc1Cl', 'O=C(CC(F)(F)F)N[C@@H]1C=C[C@@H](C(=O)[O-])C1', 'Clc1ccccc1Oc1ccc(COc2ccc3c(c2)OCCCO3)cc1', 'ClC[C@H]([NH2+][C@@H](C#N)c1ccccc1)c1csc(Cl)c1', 'Oc1cc(CC[C@@H]2CCC[NH2+]C2)ncn1', 'CN[C@H](CCc1[nH+]ccs1)c1ccsc1', 'Clc1cc(C[NH+]2CCC[C@@H]2c2ccc[nH]2)ccc1F', 'O=C([C@H](Cl)Br)[C@H]1CN2CCC12', 'Clc1ccccc1C[NH+](C[C@@H]1CCCO1)[C@H]1CC[C@](O)(COCC(F)F)C1', 'Clc1ccccc1C[NH+](C[C@H]1C[C@@H]2CCN1CCO2)C[C@@H]1CCCO1', 'Clc1ccccc1[C@H]1[NH+]=c2ccccc2=[NH+]C1=O', 'OCc1n[nH]cc1O[C@H](CN1CC[NH2+]CC1)[C@H](Cc1ccccc1)C(F)F', 'Cc1cncnc1N/N=C1\\C(=N)N(C)C2=c3ccccc3=[NH+][C@H]21', 'Clc1ccsc1[C@@H](N[NH+]1CCC[C@@H](O)C1)c1ccccc1', 'Oc1cccc([C@H]2Oc3c(F)ccc(F)c3[C@H]3[NH2+]CCC[C@H]32)c1', 'ClCc1nonc1N[C@@H]1CC[C@H]([NH3+])C1', 'O=C(c1cccn1C1CCCC1)N1CCOc2ccccc2[C@@H]1c1cc(F)cc(F)c1', 'Cc1sc(C[NH2+][C@@H]2CCCC[C@@H]2CO)c(C)c1C', 'Clc1ccc(Br)c(NC[C@@H]2NCCC[NH+]2CC2CC2)c1', 'Clc1ccc(Br)c(OC[C@H]2CC[NH+](CC(F)(F)F)C2)n1', 'COCCCNC(=O)NCC[NH+]1[C@@H](C)CCC[C@@H]1C', 'Clc1ccsc1-c1nc([C@@H]2C[NH2+]CCO2)no1', 'N=Cc1cn([C@@H]2CCC[C@]2([NH+]2CCCC2)c2ccccc2)nc1C(F)F', 'Clc1cc(C[NH2+]C[C@@H]2CCCCO2)c[nH]1', 'ClC[C@@H]1CO[C@@H](c2ccccc2)[NH+](CC2CC2)C1', 'N[C@@H]1N=Nc2nnnn21', 'Clc1cccc([S@@](=O)CCCC[NH+](C2CC2)[C@@H]2CCCO2)c1', 'Clc1ccsc1-c1cnnn1CC[C@@H]1CCCC[NH2+]1', 'O=C([O-])[C@H]1[C@H]2C=C[C@@]2(O)CCN1c1cccnc1', 'Oc1ccc(F)cc1S[C@H]1CCC[NH+](CC(F)F)C1', 'ClCc1nonc1NCCC[NH+]1C[C@H](C)CC[C@H]1Cc1ccccc1', 'CN=N[C@H]1CC(c2ccc(Cl)cc2)=[NH+]/C1=N\\OC(C)=O', 'O[C@@H]1c2ccccc2C[C@@H]1CC[NH+]1CCCC[C@H]1C1OCCO1', 'C=CCO[C@@H](C)C(=O)N[C@@H]1CC[C@@H]([NH3+])C1', 'O=C([C@H]1CC[C@@H]2CCCC[C@H]2[NH2+]1)N1CCOCC1', 'Oc1cccc([C@H]2[C@H]3[NH+]=c4ccccc4=C3CC[C@@H]2O)c1', 'Clc1ccsc1NCC[NH+]1CCCC1', 'O=C(C[NH+]1CC[C@@H]2OCC[C@@H]2C1)Nc1ccc(F)cc1F', 'Clc1cc(Br)cn1[C@@H]1CCO[C@]2(CCOC2)C1', 'Oc1cccc(Br)c1O/N=C1/C(=O)C2=C(c3ccccc3)CS[C@H]1N2', 'OC[C@@H]([NH2+][C@H]1CCOC2(CCCCC2)C1)c1ccccc1', 'Clc1cccc(-c2cn(CC(CC[NH3+])C3CCN(c4cc[nH+]cc4)C3)cn2)c1', 'Clc1ccc(Br)cc1[C@@H]1COCC[NH+]1C[C@@H]1CCCO1', 'Clc1nnnc(NN[C@H]2C[C@@H]3COC[C@@H]3C[C@@H]2C2CC2)c1', 'O=C(C1CC=CC1)N1CCC(OC[C@H]2C=N[C@H]3CS(=O)(=O)C[C@H]32)CC1', 'ClCc1cnn([C@@H]2CCCC[NH+]2Cc2ccccn2)n1', 'O=C([O-])CS[C@H]1[NH+]=c2ccccc2=C1C1CC1', 'Clc1ccc(Br)cc1/N=C(\\CCn1cccn1)N1CC[NH+]2CCCC[C@@H]12', 'Oc1cccc(S[C@@H]2CCC[NH2+]2)c1F', 'O=C([O-])[C@H](Br)[C@@H]1CCCO1', 'O=C(c1cnnn1-c1ccc(F)cc1)N1[C@@H](C(=O)[O-])C[C@]2(CCCc3ccccc32)N1Cl', 'OC[C@@H]1CC2=C(C[C@H]1O)C(=O)O2', 'Oc1cc(-n2cccc2)ncc1S[C@@H]1CC[NH2+]C1', 'Clc1cc(Br)c(C[NH2+][C@H]2CCc3nnsc32)cc1F', 'O=[N+]([O-])c1c(Cl)nsc1N1CC[NH2+]CC1', 'OCc1n[nH]cc1C[NH+]1CCC[C@H](Oc2cccc(Cl)c2)C1', 'Clc1ccc(Br)cc1-c1cnn([C@H]2CC[NH+](C3CCOCC3)C2)n1', 'Oc1cccc(SC2[NH+]=c3ccccc3=[NH+]2)n1', 'COc1nn(CC[NH+]2CCC[C@H](C(=O)NC)C2)c(C)c1Br', 'O=[N+]([O-])c1ccn([C@H](c2ccsc2)Cn2cc[nH+]c2)c1', 'Cc1c([C@@H]2CCC[NH2+][C@@H]2C)cnn1C', 'Clc1cc(Cl)nnc1O[C@H]1CC[C@@H]([NH2+]CC(F)(F)F)C1', 'Oc1cccc2nnc(S(=O)(=O)NC[C@]3(O)[C@@H]4C=CC[C@@H]4C3)n12', 'O[C@@H]1[C@@H](C(=O)[O-])[C@@H]1CC[C@@H]1CCCO1', 'Oc1cccc2cc(C[NH2+][C@@H]3CC[NH+]c4ccc(Cl)cc43)oc12', 'Clc1ccc(C[NH2+]C[C@@H]2C[C@H]3CC[C@@H]2C3)cc1', 'O=[N+]([O-])c1ccc[nH+]c1C[NH2+][C@H]1C(CO)=C(C)CC(=O)N1c1ccc(Br)s1', 'Clc1ccc(Br)cc1S[C@H](C1=c2ccccc2=[NH+]C2)C=C[C@@H]12', 'O=C(c1cccn1CCO)N1CCC(OC2CCCCCC2)CC1', 'Oc1ccccc1[C@@H]1CC[NH+]2CCCC[C@@H]12', 'COc1ncnc(N(C)[C@@H]2CCO[C@]3(CCSC3)C2)c1C', 'C[C@@H]([C@@H](O)[C@@H](c1ccncc1)n1cccn1)N1CC[NH+](C)CC1', 'CN[C@H](c1[nH+]cc(C)s1)c1ccc(Br)s1', 'Clc1cc(F)c(C[NH+]2CCCC[C@@H]2CO)cc1F', 'C[C@@H](C[NH+]1CCCCC1)N[C@H](C)c1ccc(CO)o1', 'O=C(Cc1nn(Cc2ccccc2)c2c1CCC2)N1CCCOCC1', 'Clc1ccccc1Nc1c2on(-c3ccccc3)c(=O)c2cc2c1CCCCC2', 'Clc1ccc(C[NH+]2C[C@@H]3CCC[C@@H]3C2)cc1[N+](=O)[O-]', 'Clc1cc(F)cc(NCC2(O)CC[NH+](CC[NH+]3CCCCC3)CC2)c1', 'C=CCNC(=O)[C@H]1CCC[NH+](Cc2c[nH]nc2-c2ccc(C)o2)C1', 'Clc1ccsc1[C@H]1[NH+]=c2ccccc2=[NH+]1', 'Clc1ccccc1[C@H]1[C@@H]([C@H]2CCS(=O)(=O)C2)NC2=C1C1CC[NH+](CCc3ccc(F)cc3)C=C12', 'OC[C@H](C1CC[NH2+]CC1)N1CCOCC1', 'O=[N+]([O-])c1c(O)sc2c1CC[NH+](Cc1cccc(F)c1)C2', 'O=C(c1c[nH]nc1C[NH+]1CCC[C@@H]1c1nccs1)N1CCCC1', 'O=C(/N=c1/[nH]c(C(F)(F)F)cc(-c2ccncc2)n1)[C@@H]1CCC[NH+](C2CC2)C1', 'O[C@@H](Sc1cc2c(cc1[N+](=O)[O-])CC[NH2+]C2)C(=O)[O-]', 'Clc1cc(Br)c(C[NH+]2CCN([C@@H]3CCC[C@H]3O)CC2)cn1', 'Clc1cc(Br)c(F)c(/N=C2/S[C@@H]3CSC[C@@H]23)c1', 'O=C(CNCc1nc[nH]n1)N1CC[NH+]([C@H]2CCOC2)CC1', 'Clc1cc(C[NH+]2CCC[C@@H]2CO)ccc1O', 'N[C@@H]1[C@H]2CCCC[C@H]2[NH+]1Cc1ccncc1', 'N[C@@H]1[NH2+]N2CC(c3ccccc3)=N[C@H]2C1', 'Clc1ccccc1C[NH+]1C[C@@H]2CCC[C@@H](O)[C@@H]2C1', 'C=CCNC(=O)C[NH+](Cc1cccs1)[C@@H]1CCS(=O)(=O)C1', 'Oc1cc(OCC[NH+]2CCCCC2)n[nH]1', 'ClCc1n[nH]c([C@@H]2c3ccccc3C[C@]2([NH3+])C(=O)[O-])c1', 'Clc1ccc(Br)c(N[C@H]2CCOC3(CC[NH2+]CC3)C2)c1', 'Nc1ncc([C@@H](C)[NH2+]C2CC2)s1', 'Clc1ccccc1C[NH+]1CCN([C@@H]2CCO[C@]3(CCCO3)C2)CC1', 'Clc1cc(F)c([C@@H](CF)N[C@@H]2CCOC3(CCOCC3)N2)cc1', 'Oc1cc(Br)cnc1N[C@@H]1CC[NH2+]C1', 'Clc1ccc(Br)c([C@H]2CCC[NH+]2Cc2nnco2)c1', 'Clc1cccc(CN2CC[NH+](CC[C@H]3CC[NH+](Cc4ccc([N+](=O)[O-])cc4)CC3)C2)c1', 'Clc1ccc(Br)c(C[NH+]2CC[C@@H]3CCCC[C@@H]32)c1', 'O=C([C@H](Cl)Br)C(=O)[O-]', 'Clc1ccc(Br)cc1-n1cc[nH+]c1CN1C[C@@H]2CC[C@H](C1)C2', 'Cc1sc(N)c([C@@H]2CC[NH+](Cc3cccc(F)c3)C2)n1', 'NS(=O)(=O)C[C@H]1CCCC[NH+]1Cc1cccnc1', 'O=C(C1CC1)N1CCCN(Cc2cccc(Cl)c2)CC1', 'CCN1S(=O)C[C@@]2(C(=O)N3CCCCC3)CC(=O)N=C2N1', 'O=[N+]([O-])c1c([O-])sc2nc([O-])[nH]c12', 'O=C(COCc1n[nH]c(=O)c2ccccc12)N1CCCCCC1', 'N#C[C@H](c1ccccc1)[C@H]1CCC[NH+](Cc2ccc(C(C)(C)C)o2)C1', 'COc1ncncc1C[NH+]1C[C@H](C(N)=O)[C@@H]2CCCC[C@@H]21', 'O=C(COc1cc(F)ccc1Cl)N1CCCCCC1', 'ClC[C@H](Br)[C@@]1(O)CCOC1', 'Clc1ccsc1[C@H]1CCC[NH+](Cc2ccco2)C1', 'Clc1cccc2c1NC[C@H](c1ccccc1)CN2Cc1ccccc1', 'ClCc1ccn([C@H]2CCC[NH2+]C2)c1', 'ClCc1cn(N[C@@H]2CCOC3(CCOCC3)C2)s1', 'Clc1cc(Cl)cnc1C[NH+]1CCc2oncc2C1', 'C=C(C)C[NH+]1CCCC[C@@H]1c1nc(C2CC2)no1', 'Clc1cccc2[nH]nc(N/C=C3/CS[C@@H](F)C[NH+]3)c12', 'OCS/C=C/C1=NN2CCCC[C@@H]12', 'Clc1nncnc1C[NH+]1CCOCC1', 'Clc1cccc(N2CCC([NH+]3CCC[C@H]C3[C@@H]3CCCO3)C2)c1', 'Oc1cc(O)c2nonc2c1N[C@H]1CC[NH2+]C1', 'O=C(CCOC1CCCCCC1)c1ccc(F)cc1', 'Oc1cc(OCOC[C@@]2(CBr)C3=CCC[C@@H]2C3)ccc1F', 'O=C(COCCOC1CCCCCC1)Nc1ccccc1C(F)(F)F', 'Clc1nncnc1[N-][C@H]1CC[C@@H]2CCCC[C@@H]21', 'O=C([C@H]1c2cc([N+](=O)[O-])ccc2Sc2ccccc21)[NH+]1CCN(c2cccc[nH+]2)CC1', 'Clc1cc(Cl)nc([C@H]2CCC[NH2+]2)n1', 'O=C(CS(=O)(=O)[N-]C[C@H]1CCC[NH+]1Cc1cccs1)c1ccc(Cl)cc1', 'C[C@@H](CO)C[NH2+]CCOC[C@H]1CN(C)CCO1', 'ClC(F)/C=C/C(=O)N[C@@]1(CO)CCC[C@H]1CC1CC1', 'Clc1cccc(NCCC[NH+](CCn2ccn[nH+]2)C2CC2)c1', 'O=C(c1cn[nH]n1)N(CC(F)(F)F)[C@@H]1CC[C@H]([NH+]2CCCC2)C1', 'Clc1ccccc1OCc1ccc(CN2CCCn3cc(O)cc3C2)cc1', 'O=C(CNc1nnccc1[N+](=O)[O-])Nc1ccc(CCC(=O)N2CCCCCC2)cc1', 'N=CC[C@@H]1C(=O)N[C@]2(CCc3ccccc32)C1(Cl)Cl', 'Oc1ccc(Cl)c(C(=O)N2CCCCCC2)c1', 'N=CCN1CC(=O)NC(=S)N1C[C@H]1C=CCC1', 'NS(=O)(=O)C[C@H]([NH2+]Cc1ccccn1)N1CC[NH+](C2CC2)CC1', 'O=C(c1cc(Br)cn2cnnc12)N1CC[NH+]2CCCC[C@H]2C1', 'N[C@@H](NC(=O)[C@H]1CCC[NH+](C2CC2)C1)C1(Cn2cncn2)CC1', 'Clc1ccccc1O/N=C/C(=O)Nc1ccc2[nH+]ccc(C3CC[NH+]3C)c2c1', 'OC1(C[NH2+][C@@H]2CCOC3(CCCCC3)C2)CCCCC1', 'NS(=O)(=O)N[C@H]1CC[NH+]([C@@H]2CCN(c3cccnc3)C2)C1', 'Clc1ccccc1CSc1nc2c(cnn1-c1ccccc1)c1ccccc12', 'Oc1cccc(N2CCCN(Cc3ccnc4c3cnn4-c3ccccc3)CC2)c1', 'O=C(COc1ccc(S(=O)(=O)N2CCCCCC2)cc1)NNC(=O)CCl', 'NS(=O)(=O)N[C@@H]1c2ccccc2C(=O)[C@@]12CCC[C@@H]2[NH+]C(=O)c1ccccc1Br', 'Clc1nnn(C[C@@H]2CCC[NH+](C3CCCC3)C2)c1Br', 'Clc1cc(Br)ccc1[C@H]1C[NH+]2CCCC[C@H]12', 'Clc1ccsc1C[NH+]1CCC[C@H]1Cn1cccn1', 'ClCc1nc([C@@H]2CCC[NH2+]C2)cs1', 'Clc1ccc(-c2cc[nH]n2)c([C@H]2CC[NH2+]C2)n1', 'Clc1cc(Br)cn1[C@@H]1CCC[NH+](C2CCCCC2)C1', 'O=C([O-])c1ccccc1C[C@@]1(O)C[NH2+]CCO1', 'N#C[C@@]1(SCC(F)(F)F)CSCCO1', 'Clc1ccc(Br)cc1N1CCCCCC1', 'Clc1ccsc1SC[C@@H]1CCC[NH2+]C1', 'Oc1ccc([C@H]2CC[NH2+]C[C@@H]2C(=O)[O-])cc1F', 'O[C@@H]1[C@@H](C[NH+]2CCC[C@H](CSc3cc4ccccc4[nH]3)C2)CCCN1', 'COC[C@@H]1CN(C(=O)C2=c3ccccc3=[NH+]C2)N=N1', 'CCc1cnc(C[NH+]2CC[C@@H](C)C[C@H]2C)c(C)n1', 'N#Cc1n[nH]nc1C[NH+]1CCC[C@@H]1c1cccc(Cl)c1', 'OC1=[N+]2cccc2C(=O)OC1', 'O=C(Cc1cc(N2CCCC2)ncn1)N1CCCCCC1', 'ClCc1ccc2nc(-c3ccncc3)n[nH+]2c1', 'O=[N+]([O-])c1cc(O)cc(C[NH+]2CC[C@H]3SCCC[C@@H]3N2)c1', 'Clc1ccsc1[C@H]([NH2+][C@@H]1CC[C@@H]([NH+](C)C)C1)c1ccccc1', 'Oc1cc(Br)cc2c1[C@H](N1CC[NH2+]CC1)OCO2', 'C[C@@H](NC(N)=[NH2+])[C@H]1CCOC2(CCCCC2)C1', 'NS(=O)(=O)N(C)[C@@H]1CCSc1Cl', 'C[C@@H](O)CN(C)[C@H]1CC[C@@H]([NH3+])C1', 'Nc1cnnn1/C(Cl)=N/C=C/C(=O)[O-]', 'Clc1ccc(Br)c(N2CCCCCC2)c1', 'O=C(c1cnn(CCO)c1)[NH+]1CCC[C@@H]1CC#N', 'O=C(COCC(=O)N1CCOCCC1)Nc1ccc2[nH]c(=O)[nH]c2c1', 'Clc1cc(F)ccc1C[NH2+]C1C[C@@H]2CC[C@@H](C1)C2', 'Clc1ccc([C@@H]([NH2+][C@H]2CC[C@@H]([NH3+])C2)C2CC2)cc1', 'Clc1ccc(-c2cc(=O)oc(=O)c3ccccc23)cc1', 'Clc1ccccc1NCc1nnc2n1CCCCC2', 'Clc1ccccc1OC1(C[C@@H]2CCCCCC2)CC1', 'Clc1nnc([S-])s1', 'N[C@@H](O)C[NH+]1CCC([C@H]2CCOC2)c2ccccc21', 'CCO[P@]1(=O)[C@@](C)(C(=O)OC)[C@@]1(O)C(F)(F)F', 'Clc1cc(F)ccc1/C=N/N[C@@H]1CCC[C@]12CCC[C@H]2[NH3+]', 'Clc1ccc(-c2nonc2NC2=N[C@H]3C4CCCCC43)[C@H]2C1', 'OCSC[C@H]1CCCC[C@@H]1[NH3+]', 'Clc1cccc2ncnc2c1/N=C\\c1ccc(COC2CCCC2)cc1', 'O=C(c1ccc(OCC(=O)N2CCCCC[C@@H]2C(=O)[O-])cc1)N1CCN(c2ncccn2)CC1', 'O=C(CC[C@@H]1[NH+]=c2ccccc2=[NH+]C1)N(Cc1cccs1)SC(F)(F)F', 'Clc1ccccc1-c1noc(CN2CCCOCC2)n1', 'CN=N[C@]1(C#N)CCC[C@@H]([NH+]2CCN(c3ccccc3F)CC2)C1', 'O=C(c1c[nH]nc1N1Cc2ccccc2O[C@H](c2ccc(Cl)cc2)C1)N1CCC(c2nccs2)CC1', 'ClC(/C=[NH+]/CC[NH+]1CCCc2ccc(C(F)(F)F)cc21)c1ccc(Cl)cc1', 'Clc1ccc(Br)c([C@@H]2CCC[NH+]2C[C@H]2CO2)c1', 'O=C(c1ccsc1C(F)F)N(CC(=O)N1CCCCCC1)c1ccccc1', 'O=C(c1cc([N+](=O)[O-])c2ccccc2n1)N1CCCN(c2nnc(-c3ccc(F)cc3)o2)CC1', 'Clc1cc(Cl)cc(Cl)c1C[NH+]1CCC(O)[C@H](O)C1', 'Clc1nncc(S/[N+]=C/C=CO)c1I', 'Oc1cc(OCC(=O)N2CCCCC[C@@H]2c2ccc3c(c2)OCO3)cc(OCC(F)F)c1', 'Clc1cc(/N=C/C(=O)N[NH+]2CCOCC2)cs1', 'ClCc1nnsc1[N-][C@@H]1CCS(=O)(=O)C1', 'Clc1ccsc1C1CC[NH+]([C@@H](C(=O)[O-])C(F)(F)F)CC1', 'CN[C@H](C[C@]1(O)CCSC1)C(=O)[O-]', 'OCS[C@@H]1N=c2ccccc2=[NH+]C1=O', 'O[C@@H](S[C@@H]1CCOC2(CCOCC2)C1)C(F)(F)F', 'O[C@@H]([C@@H]1CCOC2(CCOCC2)C1)[C@@H]1C[C@@H]1C', 'O=[N+]([O-])c1c([O-])c2[nH]cnc2n1C[C@@H]1C[C@@H]2CC[C@@H]1C2', 'N#C[C@@]1([NH+]2CCC(n3cncn3)CC2)CCCSC1', 'O=C(CC[NH+]1CCOCC1)N[N+](=O)[O-]', 'N[C@@H]1[C@H]([NH2+]C2CCN(c3ccc(C(F)(F)F)cn3)CC2)CCC[NH+]1C', 'Clc1ccsc1Sc1n[nH]c([C@H]2CCC[NH+]2Cc2ccco2)n1', 'Clc1ccccc1NS(=O)(=O)N1CCC(CN2CCCCCC2)CC1', 'Clc1ccc([C@@H](Br)[C@@]23CC[C@H](C[C@@H]2C)C3=O)cc1Cl', 'C=C(Cl)C[NH+]1CCc2c(F)cc(F)cc2[C@H]1C', 'Oc1cc(F)cc(C[NH+]2CCC[C@H]2c2cnoc2)c1', 'Clc1ccc(CCN[C@H]2CC(=S)N(C3CCCCCC3)C2)cc1', 'O[C@@H]1c2cc(OC(F)(F)F)[nH]c2N[C@H]1C(=O)[O-]', 'O=C(/N=C/c1ccccc1)c1cccc(C(=O)N2CCOc3ccccc3C2)c1', 'Nc1nonc1C[NH+]1CC[C@@H](OS(=O)(=O)N(CCCCO)c2cccs2)C1', 'O=C(COc1ccccc1Cl)N1CCSCc2ccccc21', 'O=C([O-])C[C@H]1CCC[NH+](C[C@@H](O)c2cc(Cl)ccc2F)C1', 'Oc1cc(F)ncc1CS[C@@H]1C=C[C@H]([NH3+])C1', 'Clc1cc(CCl)n[n-]1', 'COc1ncc2n1CCC[C@@H]2[NH+]1C[C@@H]2CCCC[C@@H]21', 'OCc1n[nH]c(N[C@H]2CC[S@](=O)C(Br)(Cl)C2)n1', 'Clc1cccc2ccc(OCCSc3nnnn3-c3ccccc3)c2c1', 'Clc1ccc(Br)cc1NC(=O)CN1CCCSc2ccccc21', 'Clc1cccc2c1Oc1ccccc1CC2', 'OCc1n[nH]cc1[C@@H]1CCC[NH2+]C1', 'N[C@@H](O)[C@@H]1CCCC[C@H]1[NH3+]', 'Clc1cc(Cl)c2c(c1)N(Cc1ccc(Cl)cc1)CCCC2', 'Nc1ncc(CC[NH2+]C[C@H]2CCOC2)[nH+]c1N', 'N=Cc1nc2ccc(N[C@H]3CC[C@@H]([NH+](CC)C)c3)cn2c1[O-]', 'Oc1cc(C[NH+]2CCSC[C@@H]2C2CCCC2)cs1', 'ClC[C@H]1CO[C@H](N2CC[NH+](Cc3ccco3)C2)CC1', 'Oc1ccccc1CC(=O)N1CCOCCN1c1ccccc1', 'Clc1ccc(C(=O)NNC(=O)N2CCCCCC2)cc1F', 'Clc1ccsc1CN1CCN(C(=O)NC2CCCCCC2)CC1', 'O=C(/N=C\\NC1=CN=NC1)\\CC(=O)[O-]', 'O=C([O-])[C@H]1[C@H]2C=C[C@@H]1[C@@H](c1ccc(OC(F)F)cc1)N2', 'C[C@@H](OCC[NH3+])[C@H]1C[NH2+]CC[C@@H]1c1c(F)cccc1F', 'Clc1cc(F)cc2c1CCN(C(=O)Cc1ccc(OC(F)F)cc1)CC2', 'O=C([C@H]1[C@H]2C=C[C@@H](C2)[C@H]1C(=O)[O-])C1CCCCC1', 'Clc1cc(CSC2=NC[C@H]3CS(=O)(O)C[C@H](O)[C@H]23)ccc1O', 'ClCc1csc([N-]S[C@H]2C=CCC2)n1', 'CN[C@H]1C[C@@H](C)N=C(C)[C@H]1O[C@H]1C[C@H](C)[C@H](O)C1', 'O=C(c1c[nH]c2ncn(Cc3ccccc3)c12)N1CCCCCC1', 'Clc1cc(/N=C\\c2c[nH]c[nH+]2)cs1', 'ClC(c1cc2cccc(Cl)c2o1)[C@H]1CCC[NH+]2CCC[C@@H]12', 'Oc1cc(Br)c2ccccc2c1C(=O)N1CCCCCC1', 'Clc1nncc(C[NH+]2CCC[C@@H]2c2ncc[nH]2)n1', 'C[C@@H]([NH2+][C@@H]1CCN(CCn2cc[nH+]c2C(C)(C)C)C1)c1nccs1', 'O=C(COc1cc(C2(O)CCCCCC2)cc(Br)c1)NCC1CCCCC1', 'Clc1cc(Nc2ncnc3c2CCCCC3)ccc1Cl', 'Clc1cccc2nc(Nc3ccc4c(c3)OCO4)nc2c1', 'Oc1cccc(CN(CCN2CCOCC2)C(=S)N2CCCCCC2)c1', 'Clc1ccc(-c2cc(F)c(=O)[nH]c(=O)n2)cc1', 'O=[N+]([O-])c1ccc2[nH]ncc2c1C[NH+]1CC[C@H](C[NH+]2CCCC2)C1', 'Clc1cccc2ccc(Nc3ccccc3C(=O)OC[C@H]3CCCO3)c2c1', 'Oc1cc(C[NH2+][C@H]2CCOC3(OCCCC3)C2)ccn1', 'Clc1nnc(C[NH+](C2CC2)[C@H]2CCS(=O)(=O)C2)s1', 'C=CC[NH+](CC(=O)[O-])CC1CCCC1', 'COC(CNC(=O)c1cc2ccccc2oc2ccccc12)c1ccccc1', 'N#CCCC[NH+](CCCBr)CC(F)(F)F', 'Clc1ccc(C[NH+]2CCC[C@]2(C(=O)[O-])[C@@H]2CCOC3(CCOCC3)C2)cc1', 'NS(=O)(=O)C[C@@H]1[NH2+]CC[NH+](Cc2ccccc2)C1', 'Clc1ccc([C@@H](OC[C@H]2CCC[NH2+]C2)[NH+]2CCCC2)cc1', 'Clc1cccc2c1N(Cc1nccn1Cc1ccccc1Cl)CCCC2', 'ClCc1cn(C2CCCCCC2)nn1', 'Clc1cccc([C@@H]2C[C@@H]2C[NH+]2CCC(n3cc[nH+]c3)CC2)c1', 'O=[N+]([O-])c1cccc2c1/C(=N/CCCOc1ccccc1)CCCC2', 'Oc1ccc(F)c([C@@H](C[NH2+]C(N)=O)[NH+]2CCCC2)c1', 'Clc1ccc(-c2nonc2N2CCc3ccccc3CC2)cc1', 'CNc1cc[nH+]c(N2C[C@H]([NH3+])C(=O)CC2)c1', 'CCS[C@@H]1CCCCN(Cc2ccccc2N2CCCC2)C1', 'N[C@@H](CCC(F)(F)F)C(=O)N1CC[NH+]2CCCC[C@@H]2C1', 'Clc1cccc2c1C[NH+](C[C@@H]1C(=O)NCC[NH+]1Cc1ccccc1Cl)[C@@H]1CC=C[C@H]21', 'O=C(CC[C@@H](O)c1ccccc1)N[C@H]1CCCOc2ccccc21', 'Oc1ccc(O[C@H]2C=[NH+]N=C3CCC[C@H]32)cc1C[NH2+][C@H]1CCS(=O)(=O)C1', 'NS(=O)(=O)/N=C\\C1=CCCC[NH2+]1', 'Clc1ccc(CN2NCCN(c3ccccc3C(=O)[O-])CC2)cc1', 'Clc1ccc(CNC(=O)N2CCCCC[C@@H]2c2ccccc2)o1', 'Clc1ccsc1[C@H]1[NH+]=c2ccc(O)cc2=[NH+]C1=O', 'CC[C@@H](COC)[NH+]1CC2(CC[NH+](Cc3ccccc3)CC2)C1', 'Oc1ccc(F)c(-c2nc(C(=O)N3CCCN(C4CC4)CC3)cnc2-n2cccc2)c1', 'ClC[C@@H]1OCCC[C@@H]1[NH2+]C1CC1', 'O=C(C[NH+]1CCCN(c2cc(F)cc(F)c2)CC1)Nc1ccccc1C(=O)NCc1ccccc1', 'Clc1ccsc1[C@@H]1[NH+]=c2ccccc2=[NH+][C@H]1COC(F)(F)F', 'O=C(c1cccc(Oc2ncnc(-n3cccc3)c2)c1)N1CCCCCC1', 'C=CCC1CC[NH+](Cc2cnc3[nH]c[nH+]c3c2)CC1', 'O=C(c1ccc(S(=O)(=O)N2CCOCC2)cc1)N1CCN(c2nc3ccc4ccccc4c3c(COc3ccccc3)nc2=O)CC1', 'Clc1ccccc1O/C=C/CN1CCCSCC1', 'N[C@@H]1c2ccccc2C[NH+]1C/C([O-])F', 'O=C(c1cn2ccsc2n1)c1ccc(S(=O)(=O)N2CCCCCC2)cc1', 'C=C[C@@H]1CC[C@](C#N)([C@@]2(O)CCC(C)(C)[C@@H]2O)[C@@H]1O', 'O=C(c1ccc[nH]c1=O)N1CCO[C@@]2(CCS(=O)=C2CO)N1', 'C[C@@H]([NH2+][C@@H]1CC[C@@H]([NH+](C)C)C1)c1cccc(Cl)c1', 'Oc1ccc(F)cc1-c1onc(N2CCCCCC2)n1', 'Clc1nnn(-c2ccc(F)cc2)c1CN1CCCCCC1=O', 'Clc1ccccc1[C@H](c1ccccc1)n1c([O-])nnc2ncnc2c1=O', 'N#C[C@H]([C@@H]1O[C@@H](O)[C@H]1CC(N)=O)[C@@H]1CCCC[C@H]1C(=O)[O-]', 'COc1nn[nH]c1/C=C/C(=O)Nc1ccc(N2CCCCCC2)cc1', 'Nc1[nH]ncc1C[NH+](Cc1ccsc1)C[C@@H]1CCC[NH+](C2CCCCC2)C1', 'Clc1cc(CNCC[NH+]2CCC[C@H]2C[NH+]2CCCC2)cnc1C1CC1', 'ClC(F)(C[C@@H]1CCO[C@]2(CCSC2)C1)C(F)(F)F', 'Oc1cccc(OCc2nc(C3CCCCCC3)[nH]n2)c1', 'CNS(=O)(=O)[C@H]1CC[NH2+]C[C@@H]1C(C)C', 'ClCc1noc(Cn2cnnn2)c1N1CCCN(c2ccccc2)CC1', 'OCc1ncnc(OC2CCCCCC2)c1N', 'Clc1ccsc1-c1csc(N2CCCCCC2)n1', 'N#C[C@H]1C(N)=N[C@H]([C@H]2CC[NH2+]C2)c2cc(Br)ccc21', 'Clc1ccccc1Cn1cnc2ccccc2c2nc(N3CCCC3)ncc21', 'Clc1cc(Cl)ccc1CN1CCCCC[C@@H]1c1ccccn1', 'Clc1nnnn1CC[NH+]1CCC[C@H]1c1c[nH]cn1', 'O[C@@H]([C@@H]([NH3+])CO)C(F)F', 'Clc1cc(Br)cn1C[C@@H]1CCCCCC1', 'CN[C@H](CN1CC[NH2+]CC1)Cn1cc[nH+]c1C', 'OC1CC[NH2+]CC1', 'N#C[C@@]1(O)CCc2c(s1)C[NH+](Cc1csc(-c3cccs3)c1O)CC2', 'Clc1ccccc1Oc1cncc2ncnc2c1O', 'Clc1nncnc1[NH2+]C[C@@H]1C=C[C@H](C(=O)[O-])C1', 'ClC(=S)O/C=C1\\C(/C=N\\C(=O)[O-])C(=S)NC(=O)N1', 'OCS1(=O)CCCCC1', 'Clc1cc(Cl)ncc1-c1cccc2c1OCCCO2', 'O=C(CN(c1cncc(C(F)(F)F)n1)S(=O)(=O)c1ccccc1)N1CCCCCC1', 'Oc1cc(Cn2ccc3cc(Cl)ccc32)c([N+](=O)[O-])ccc1=O', 'O=[N+]([O-])c1c(Cl)nsc1N[C@@H]1CC[NH+]2CCCC[C@@H]12', 'Oc1ccc([C@H](CN2CCCCCC2)c2ccccc2)c(F)c1', 'CN(c1cccc(Cl)c1)S(=O)(=O)c1c(C(N)=O)cnc2ccccc2c1=O', 'O=C(c1ccsc1C1CC1)N1CCCCC[C@@H]1c1ccc(F)cc1', 'CN=C1C(=O)N=C(N2CC(=O)N(Cc3cccnc3)C2=O)N=C(N)N1c1ccccc1', 'N=Cc1cc[nH+]c(N2CCCC2)c1N[C@H]1CCO[C@]2(CO2)[C@H]1C', 'Clc1nnc(/N=C/c2ccc3c(c2)OCCCO3)c(-c2ccccc2)n1', 'O=C(c1ccsc1)c1nnn2c1Nc1ccc(Cl)cc1C(=O)N2CCc1ccccc1', 'Clc1ccc([C@@H](C[NH+]2COc3ccccc3OC2)c2ccccc2)cc1', 'ClC[C@H](O)C1CC[NH+](C2CC2)CC1', 'COCc1c(N)[nH+]c(C)n1CC[NH+]1CCC[C@H]1CC', 'ClCc1cnc(COCCCOC2CCCCCC2)s1', 'Clc1cc(Cl)nnc1N[C@H]1C[C@H]2CC[C@@H](C1)[NH2+]2', 'Clc1cccc2cc([C@H]3CCCCCN3C(=O)Cn3cncn3)ccc12', 'O=C(/N=C/CC(=O)Cc1ccccc1)[C@@H]1CN(Cc2ccccc2)CCCO1', 'Clc1ccsc1Sc1cc([N+](=O)[O-])ccc1N1CCCCCC1', 'Clc1ccc(Br)cc1Sc1nc(NC2CCCCCC2)n[nH]1', 'Clc1cc(F)cc2nc(CN3CCCCCC3)nn12', 'O=[N+]([O-])c1ccc2cc(=O)oc(=O)c(-c3cccc([N+](=O)[O-])c3)c2c1', 'CNS(=O)(=O)c1csc(C(=O)N[C@@H](C)C(=O)N2CCCCCC2)c1', 'ClCc1csc2s1CCCC2', 'Oc1ccccc1[C@H]1Oc2ccccc2Oc2ccccc21', 'COC[C@H]1CCC[NH+]([C@@H](C)c2[nH+]ccn2CC)C1', 'O=C(/C=C/c1c(NN)cc(C#N)c(=O)c2ccccc12)Nc1ccc(Oc2ccccn2)cc1', 'OCc1ncn[n-]1', 'Oc1cccc2cc(/C=C(\\Cl)c3ccccc3)c(=O)c(=O)[nH]c12', 'Oc1cc(F)cc(C[NH2+][C@@H]2CC[NH+](CC3CC3)C2)n1', 'O=C([C@H](CC1CC1)N1CCOc2ccccc2C1)OC[C@@H]1CCCCO1', 'Clc1ccccc1[C@H]1NC(=O)COc2c1ccc1c2cccc1OCc1ccco1', 'Clc1cccc2c1OC[C@@H](c1ccccc1F)CN2Cc1ccccc1Cl', 'O=C(/N=C/C1=C(N)NC(=O)N(c2ccc(Cl)cc2)c2ccccc21)c1ccccc1', 'O=C(COCc1nnc2n1CCN(Cc1ccccc1Cl)CC2)N[C@@H](c1ccc(Cl)cc1)C(F)(F)F', 'Clc1cc(C(F)(F)F)ncc1N1CCCSCC1', 'O=C(/C=C\\CS(=O)(=O)N1CCCSCC1)Nc1cc(Br)ccc1O', 'O=[N+]([O-])c1cc(SC2CCN(c3ccccc3)Cc3ccccc32)c2[nH]ccc2c1', 'Oc1cc(-n2cc[nH+]c2)ccc1NC(=O)[C@@H]1C[C@H]2C=C[C@@H]1C2(Cl)Cl', 'O=[N+]([O-])c1ccnc2c1[nH]c(=S)n2C1CCCCCC1', 'Clc1cccc2nnnc2c1COc1cccc2c1OCO2', 'N[C@@H](c1cccc(Br)c1O)[C@@H]1C[NH+]2CCN1CC2', 'Clc1cccc(N2CCC/C(=C/c3cccc(C(F)(F)F)c3)SC2=O)c1', 'COC[C@]1(C)CCCN(C(=O)C(=O)Nc2ccccc2)CC1', 'Clc1ccc(-c2nnc(-c3ccc(=O)ccc3CO)s2)c(O)c1', 'ClC[C@H]1CNCC[C@H]1C[NH+]1CCCC1', 'Oc1ccc(O[C@H](C(=O)NC2CCCCCC2)C(F)(F)F)cc1', 'Clc1ccccc1OCC1NCCN(C(=O)Cc2ccc(C#N)cc2)CC1', 'CN(CCOc1ccccc1)C(=O)CSc1nnc(SCC(N)=O)c(=S)c2ccccc12', 'Clc1ccccc1N1Nc2c(cc(N3CCCC3)c(=O)c3ccccc32)CCC1', 'O[C@@H]1COCC[C@@H]1[NH2+][C@@H]1CCOC2(CCC2)C1', 'O=[N+]([O-])c1c(F)cccc1CN1CCCCC[C@H]1c1cccc(F)c1', 'Oc1ccc(/C=N\\N=C2\\NCCS2)cc1N1CCCCCC1', 'C=CCC[NH+](CC(C)C)[C@@H](C)C(=O)[O-]', 'Clc1ccccc1Nc1nnc2c(Br)ncncc12', 'Clc1ccsc1/C=[NH+]/C[NH+]1CCC2(CCOCC2)C1', 'O=C(/N=c1/[nH]c/nc2ccc(Br)cc2c1=O)c1ccc(C2CCCC2)cc1', 'Clc1ccccc1Oc1cncc(/C=C/N2CCN(CCC[NH+]3CCCCCC3)CC2)c1', 'C=C(C#N)[C@@H](C#N)C[NH+](C)C/C(C)C', 'Clc1ccccc1CC1(C[NH2+][C@@H]2CCCc3ccccc32)CCCCCC1', 'O=C([C@H]1[C@H](c2cccc(Br)c2)C[C@@H]1c1ccccc1)N1CCCN(CC(=O)[O-])CC1', 'ClCc1nnn[n-]1', 'Oc1cc(F)cc2[nH]ncc2c1=O', 'C[C@@H](O)C[C@@](C)(O)CNC[C@@H]1CCC[NH+]1C', 'O=C(c1c[nH]nc1CN1C[C@H]2CCC[C@]21[NH2+]C1CCCCC1)N1CCC(C(=O)N2CCCCCC2)CC1', 'ClCc1nnsc1NC1CCCCCC1', 'ClC[C@@H]1Oc2ccccc2Oc2ccccc21', 'O=C([C@H]1CCCC[C@@H]1C[NH3+])[NH+]1CCCC1', 'Clc1ccccc1Nc1nc(Cl)nc(NN2CCOCC2)c(=O)c1C#N', 'O=[N+]([O-])c1ccc2c(c1)CCN(Cc1ccc([O-])cn1)CC2', 'O=C(c1c[nH]c2cccc(F)c12)N1CCC[NH+](Cc2ccccc2)CC1', 'O=C([C@H]1c2ccccc2CCc2sccc21)N1CCOCC1', 'Clc1ccc([C@@H](Cl)CC[C@@H]2CCCCN(S(=O)(=O)c3ccccc3)C2)cc1', 'CNC(=O)[C@H]1[NH2+]CC[C@H]1C(=O)[O-]', 'ClC([O-])=CCC[C@H](c1ccccc1)N1CCCCCC1', 'Clc1cccc2c1N[C@H](c1ccc(NCc3ccc(Cl)cc3)[nH]c1=O)CC(O)(CO)C2', 'O[C@@H]1[C@@H](CNC2CCCCC2)CCCN(Cc2ccco2)C1', 'Clc1ccccc1[C@H]1Oc2ccc(Cl)cc2CCN1c1ccc([N+](=O)[O-])cc1', 'OC1=Nc2ccccc2C[C@@H](c2ccccc2)C1', 'O=C(CS(=O)(=O)c1c(F)cccc1F)NC[C@@H](O)C1CCCCCC1', 'ClC[C@H]1O[C@H]([N+]2CC2)C1(Cl)CCO', 'ClCc1cc(-n2cccc2)cc2c1NCCCO2', 'Oc1cc(CCN[C@@H](c2ccc(F)cc2)c2ccc3c(c2)NC(=O)CCC3)ccn1', 'ClCc1cccc(F)c1OC[C@@H]1CCCCC[C@H]1CO', 'Oc1ccc(/N=c2/sccc(=O)n(Cc3ccc4c(c3)OCO4)c2=O)cc1', 'Oc1cc(-n2nnnc2COCC2CC2)cc(OC2CCCCC2)c(C(F)(F)F)c1=O', 'Clc1nnn(Cc2ncnn2C2CCCCCC2)c1Cl', 'Oc1cc(F)c([C@@H](O)CN2CCCCCC2)c(Cl)c1', 'NS(=O)(=O)/N=c1\\sc(N)c(-c2ccc(Br)cc2)c(=O)c2ccccc12', 'Clc1cccc(C2SCCCOC2)c1Oc1ccccc1', 'O=C(c1c[nH]cn1)N1CCCSCC1', 'Clc1ccc(-c2n[nH]c3cc(C(=O)NCC[NH+]4CCCCCC4)ccc23)cc1', 'OCSC[C@@H](CO)[C@@H]1C[NH2+]CCO1', 'O=[N+]([O-])c1c(-c2cc(O)ccc2[N-]S(=O)(=O)c2ccc(Br)cc2)sc2c1CCCCC2', 'Clc1cccc(-c2nnc([C@H]3CCCN(S(=O)(=O)CC4CCCCCC4)C3)o2)c1', 'CNS(=O)(=O)c1c(C)c(C)nn1CC(=O)N1CCC[C@@H](C)CC1', 'Clc1ccc(-c2cc(N[C@@H]3CCS(=O)(=O)c4ccccc4C3)ccn2)cc1', 'N=C1/N[N+](=O)OC1', 'O=C(CSC(F)(F)F)N1CCCSCC1', 'CC[C@@H](C[C@H]1CCCC[NH+]1C)C(=O)[O-]', 'Clc1nncnc1[N-][C@@H]1CCC[NH+](CC(F)(F)F)C1', 'C=C[C@@H]1N=C2O/N=N[C@H]2C(=O)[C@@H]1C', 'NS(=O)(=O)/N=C/C[C@@H]1CCC[NH2+]1', 'ClC[C@@H]1OCCOc2ccccc21', 'Clc1ccc([C@@H](Cl)[C@@H]2CCCOCC2)cc1', 'Clc1ccc(-c2csc3[nH]c(COCN4CCOCCC4)nc23)nc1', 'Clc1cc(Br)c2nc([C@H](c3ccc(F)cc3)[NH2+][C@H](c3ccco3)N3CCCCCC3)sc2c1', 'Clc1nncc(C[C@@H]2CCCCc3ccc(F)cc23)c1C1CC1', 'Oc1cc(Br)cn/c(=[N+]([O-])c2ccccc2)c1', 'Oc1ccc(Oc2cc3occc3nc2C(=O)N[C@H]2CC=CCCC2)cc1', 'Clc1cccc2c1CNC(=O)C[C@@H]2c1ccccc1C(F)(F)F', 'Clc1nnc([S-])c2c1N=C1C[C@H](Cl)CC1(CC[NH+]1CCOCC1)C2', 'Clc1ccccc1-c1csc(C[NH2+]C2CCCCCC2)n1', 'Clc1nncc(Oc2ccc(F)cc2)c1N[C@H]1CCCOc2c(Cl)cccc21', 'OC[C@@]1(O)CCc2ccccc2CN1c1ccc(F)cc1', 'O=[N+]([O-])c1c([N+](=O)[O-])c2c([nH]1)C[C@@H](C(=O)N1CCCCCC1)CCC2', 'O=[N+]([O-])c1c(OCCN2CCCCCC2)cnn1C(Cl)F', 'COC(=O)[C@@]1(F)CC[NH+](CC(=O)[O-])C1', 'CNS(=O)(=O)[C@@H]1CCOCCN1C(=O)/C=C/c1ccc(Br)cc1', 'Clc1ccccc1NS[C@H]1NC(=O)c2cc(C(=O)NC3CCCCCC3)ccc21', 'C=C[C@@H]1[C@@H](C)CC[C@@]1(O)C[NH+]1CCCC1', 'O=[N+]([O-])c1c(O[C@@H]2CCCCCC2=O)cc(N2CCOCC2)c2ccccc12', 'Clc1nnn(C(F)F)c1C(=O)Nc1ccc2c(c1)N(C)c1ccc(Cl)cc1C(=O)N2', 'Oc1cccc(Nc2ccc(F)c3c2[nH]c2ccccc3c2=O)c1[O-]', 'O=[N+]([O-])c1cc(F)ccc1N1CCC([NH+]2CCCc3cc(F)ccc32)Cc2ccccc21', 'Clc1ccccc1[C@H]([NH3+])[C@@H]1C[NH+]2CCN1C[C@@H]2C', 'O[C@@H]1C[C@@]2(OC(=O)C[NH+]3CCCC[C@@H]23)C1', 'ClC[C@H]1CC[C@@H]([NH+](CC(=O)[O-])CO)C1', 'Clc1ccccc1CSC[C@@H]1CCCCC[C@@H]1C(=O)[O-]', 'Cc1nnsc1C(=O)N1CCOCC[C@@H]1C', 'O=[N+]([O-])c1ccc2cccnc2c1C[NH+]1CCC2=C(N2c2ccccc2)CC1', 'O=[N+]([O-])c1cccc2c1OCCCN2[C@H]1CCS(=O)(=O)C1', 'O=C(COc1ccc2nsnc2c1)N[C@@H]1CCCC[C@H](O)C1', 'Oc1cc(COC[C@H]2CCCCOC2)cc([N+](=O)[O-])c1Cl', 'ClCc1cnn([C@H]2CCCCC[C@@H]2C(=O)N2CCCC2)c1', 'Clc1cccc(Nc2cn(Cc3ccccc3)nc2C[NH+]2CCCCCC2)c1', 'Clc1ccccc1CC[NH+]1CCCN(Cc2cccc(F)c2)CC1', 'O=C(c1c[nH]nc1-c1ccsc1)N1CCN(C(=O)[C@@H]2CCCc3ccccc3C2)CC1', 'ClCc1cn[nH]c1N1CCCSCC1', 'O=C(Cc1n[nH]c2c1CC[NH2+]CC2)N1CCc2ccccc21', 'O=[N+]([O-])c1ccc2[nH]ncc2c1N[C@H]1CCCOc2ccc(F)cc21', 'OC[C@@H]1[C@H]([NH2+]C2CC2)CCC[C@@H]1CC[NH+]1CCCCC1', 'O=C(CNc1nn(CC[NH+]2CCCCCC2)c2c1CCCC2)Nc1ccccc1', 'O=[N+]([O-])c1c(Cl)nnc(NC2(C#N)CCOCC2)c(-c2ccc(Cl)cc2)c1=O', 'Nc1n[nH]nc1COC(=O)[C@@H](C)N1CCCCCC1=O', 'Oc1ccc(F)cc1[C@H]1Oc2ncccc2Oc2ccccc21', 'Clc1ccccc1-c1cnn2c1N=C1CCCC[C@H]1CC2', 'Oc1cc(C=O)c2sccc2c(=O)c1Br', 'CN[C@H]1[C@H](OCC(C)C)CCCc2cccc(Br)c21', 'Cc1cn([C@@H]2CCCCCN2c2ccccc2)c(C#N)c1N', 'Oc1ccccc1[C@H]1C2C[C@@H]3CCCCC[C@H]3C[C@@H]21', 'CNC(=O)[C@H]1CCC[NH+]1CC(C)(C)[NH2+]C', 'N=CC[C@@H](O)C[NH+]1C[C@H](C)C[C@@H](C)C1', 'O=[N+]([O-])c1cc(Br)ccc1OCCN1CCC[NH2+]CC1', 'ClC[C@@H]1CC[NH2+]C1', 'Oc1ccc([N+](=O)CN2C(=O)COc3ccccc3[C@@H]2Nc2ccccc2)cc1', 'Clc1cc(Cl)cnc1N1CCCCC[C@@H]1C[C@@H]1CCCO1', 'Oc1ccc(F)cc1CN1CC[NH+]c2ccccc2c1=O', 'N=Cc1cccc2c1OCCN(C(=O)/C=C\\c1nc(C)co1)C2', 'O=C(c1c[nH]nc1-c1ccco1)N1CCC[NH+](Cc2ccccc2)c2ccccc21', 'Clc1ccc([C@@H](CC2COc3ccccc3CC2)NCCc2cccnc2)s1', 'O=C(c1c[nH]nc1Nc1nsnc1C(F)(F)F)N1CCCN(C(=O)C(F)(F)F)CC1', 'Clc1cccc(-c2nnc(SCCC[NH+]3CCCCCC3)nc2-c2ccccc2)c1', 'O[C@@H](c1ccccc1Cl)[C@H]1CSCCSC1', 'COCCOc1ccc(CN2CCC[NH2+]C3(CCCCC3)C2)cc1', 'C=C1c2ccc(Cl)cc2CCN(C(=O)[C@H]2CC=CCC2)C1', 'Clc1cccc(CCC[NH+]2CCCCCC2)c1', 'Clc1ccccc1NCCN1CCC[NH+](C2CCCC2)CC1', 'O=C(c1cn2ccsc2c1N[C@@H]1CCOC1)N1CCC[C@@H](c2nnc3n2CCCCC3)C1', 'O=C(c1cnn2c1NCCCC2)N1CC(c2ncon2)C1', 'O=C(/C=C1/C[C@H](c2ccncc2)C(=O)Nc2c1CCCC2)N1CCOCC1', 'ClC[C@@H]1Oc2ccc(F)cc2[C@@H]1C1CCCCCC1', 'Clc1ccc(-c2nn(-c3ccc(F)cc3)c3c2NCCC[C@H]3OC(F)(F)F)cc1', 'Oc1cccc(S[C@@H]2NC(=O)COc3ccc(Cl)cc32)c1', 'Clc1ccc(CC[C@H]2C[C@H]3CC[C@@H]2CN(C(=O)NCCC(=O)N4CCCC4)C3)cc1', 'Clc1ccccc1N1NC2=NC=C2OC(=S)N1C1CCCCCC1', 'Oc1cc(Br)c2s/c(=C\\c3ncc(C(F)(F)F)o3)c(=O)c3ccccc3c2c1O', 'Oc1cc(OC[C@@H]2CC[C@@H](C(F)(F)F)N2)c2ccccc2c1C1CCCCCC1', 'Clc1ccccc1-c1noc2c1N=C(SCc1ccccc1)C(=O)C[C@@H]2c1cccc(F)c1', 'ClC(=S)Sc1ccc(N[C@H]2CCCC[C@H](C(=O)[O-])C2)cc1', 'ClC[C@@H]1OCC[NH+]1[C@H]1CCOC2(CCSCC2)C1', 'Clc1cc(C2(C(=O)[O-])CCCCCC2)no1', 'Clc1ccccc1-c1nc(CO)[nH]c2cc(-c3ccco3)[nH]c2c1N', 'O=C(CCCOc1ccccc1Br)N1CCCCCCC1', 'O=C([O-])[C@H](Cc1ccc(F)cc1)NC=C1C[NH2+]CCc2ccccc21', 'Clc1ccccc1[C@H]1[C@@H](n2cccn2)CCC[NH+](CCc2ccccc2)C1', 'Oc1ccc([C@H]2C(C#N)=C(Nc3ccccc3)NC(=O)N(c3ccc([N+](=O)[O-])cc3)N2)cc1', 'N#C[C@@H]1CCCN(C(=O)[C@H]2C[C@@H]2c2ccc(F)c(F)c2)CC1', 'Clc1ccc(-c2nnnn2CCC[NH+]2CCCCCC2)nc1', 'O=C(c1c[nH]c([N+](=O)[O-])c1)Oc1cc2c(cc1Cl)OCCC[C@@H]2O', 'Oc1ccccc1C1NC[C@@H](c2cccs2)Cc2cccnc21', 'C[C@@H](OC(=O)[C@@H]1CCCO[C@H]1C)c1ccc2c(c1)OCCCO2', 'N#C[C@@H]1COCCCN1Cc1ccc(Br)s1', 'Clc1ccc(Br)cc1/N=C/C(=CC1=c([O-])c([O-])c(Cl)c1=O)N1CCCCCC1', 'Clc1ccccc1NS(=O)(=O)NCC[NH+]1CCCCCC1', 'O=C(c1ccco1)[C@H]1Sc2ccccc2NC(=O)[C@H]1Cc1cccc2ccc(Cl)cc12', 'O=C(CSCNO[C@@H]1CCCOc2ccccc21)N[C@H]1CCc2ccccc21', 'O=[N+]([O-])c1ccc(CN2CCc3[nH]c[nH+]c3/C(=N/OCc3cccs3)C2)cc1', 'Clc1ccc(-c2cc(F)c([C@@H]3CCCN(c4nccs4)c3)CCN2)cc1', 'OC[C@@]1([C@H]2CCC[NH2+]C2)CCC[NH2+]C1', 'N[C@@H](NCc1c[nH]nc1-c1ccccc1)C(=O)N1CC[C@H]2[C@@H](CCCc3ccc(F)cc32)C1', 'Clc1cccc(COc2cccc(/C(Nc3ncccn3)[NH+]3CCCCCC3)c2)c1', 'OC[C@H]1OCC[C@H]2CCCC[C@@H]2C1', 'COc1ncnc(Sc2nn3c(=O)c(C)c(C)nc3c(=O)c3nc(C)sc23)c1N', 'NS(=O)(=O)C[C@H]1Nc2cc(Cl)ccc2N2CCCCC[C@@H]12', 'O=C(C[NH+]1CC[C@@H]2CCCCC[C@@H]21)Nc1ccc(I)cc2cccc12', 'Cc1c([C@@H](C)CCCCO)c(=O)n2c(-c3ccc(Br)cc3)n[nH]c2nn1C', 'C[C@@H]([NH2+]C1CCC(N2CCOCC2)CC1)c1ncnc2c1CCCCC2', 'Clc1cccc(Br)c1NC1=NS(=O)(=O)CC=CC1', 'ClCc1n[nH]c2c1CCC[C@H]([NH2+]CCc1ccccc1)C2', 'O=[N+]([O-])c1c(F)ccc2ncn([C@@H]3CC(=O)Oc4ccccc4N3)c12', 'Cc1ncc2cc(C(=O)N[C@H](C#N)C(N)=O)cc(=O)cc2n1', 'Clc1ccsc1N1CCCCC[C@@H]1C(=O)[O-]', 'Nc1nc([C@H]2CCCc3cccnc3N2)no1', 'Clc1ccc(-c2nonc2N2CCC[NH2+]CC2)cc1Cl', 'O=C(c1ccsc1)[C@@H]1C[C@@H](c2ccccc2)[NH2+]Cc2ccccc21', 'Clc1cccc2ncncc1n2CCOc1ccccc1', 'Oc1cc(O)ccc1S(=O)(=O)N1CCC[NH+](Cc2ccc(F)cc2)CC1', 'O=C([C@H](c1ccc([O-])c(C(F)(F)F)c1)N1CCCCCC1)N1CCCS1(=O)=O', 'Clc1cc(Cl)cn2c(-c3ccco3)cccc2n1', 'Clc1cc(F)cc2ncn(CCC[NH+]3CCCCCC3)c12', 'O=[N+]([O-])c1c([O-])n2ncc(C3CC3)c2ncn1-c1ccc(F)cc1F', 'O=C(CSCc1nnc2n1CCC2)NC1[C@@H](C#N)CCCc2ccccc21', 'Clc1ccccc1[C@H](Br)C1(C[NH+]2CCCCCC2)CC1', 'Clc1nnnc(S[C@@H]2CCCOc3ccc(OC(F)(F)F)cc32)c1Br', 'Oc1cc(-n2cnc([C@@H]3CCCN(c4ccccc4OC(F)F)c3)ncn2)ccc1O', 'Clc1cc(/C=N/O[C@@H]2CCCCNC2)cs1', 'O=[N+]([O-])c1c(Cl)nsc1N[C@H]1C[C@@H]2C(=O)CCN(Cc3ccccc3)C[C@H]21', 'OC[C@H]1O[C@H](O)CS[C@H](OCc2ccccc2)C1', 'Oc1ccc([C@H]2[NH2+]C[C@H](Nc3ccc(S(=O)(=O)N4CCOCC4)cc3)CSC2=O)cc1F', 'Clc1cc(NC(c2ccc3c(c2)OCCO3)[NH+]2CCCCCC2)ccc1Cl', 'Clc1cccc2c(N3CCC[NH+](CC4CC4)CC3)ncnc12', 'Clc1ccc(-c2n[nH]cc2C[NH+]2CCCN(Cc3ccc(F)cc3F)CC2)cc1', 'Clc1cccc2cccnc1n2CCNc1ccccc1', 'C=CC[NH+]1CCC(CCC(=O)N2CCCCCC2)CC1', 'Clc1ccc(-c2ccccc2N2CCC[NH2+]C[C@H]2CS(=O)(=O)[O-])cc1', 'O=[N+]([O-])c1c(Cl)nc(C2CCCCC2)ncn1C1CC1', 'O[C@@H]1[C@@H]([C@@H](O)C[C@@H]2CCCOCC2)c2cc3ccccc3nc21', 'N=C1\\[C@@H](SCCn2c(N)nc3ccccc32)[C@H]2C=CCCCN12', 'O=[N+]([O-])c1cc(C[NH+]2CCC[C@@H](CN3CCOCC3)N(c3ccccc3)C2)ccn1', 'Cc1nnc(SCCN2C(=O)N[C@]3(CC[C@@H](O)CCC3)C2=O)s1', 'O=C([C@H]1CO[C@H]1c1ccccc1)N1CCCN(c2nc3ccccn3[nH+]2)CC1', 'O=C(Cc1cc2n(n1)COCCO2)[C@H]1Cc2ccc(F)cc2O1', 'Clc1cccc2nc(N3CCC[C@@H](CC4CC[NH+](C5CCCCC5)C4)CC3)nn12', 'Clc1ccccc1-c1cnn2c1NCCC[C@@H]2[NH2+]CCC(C)C', 'Clc1cccc2cccnc1n2CCC(=O)N[C@H](c1ccccc1)C1CCCC1', 'Clc1ccc(-c2nnnn2/C=C/CC[NH+]2CCCCCC2)nc1', 'C=C1[C@H]2CC[C@@H]1CN(Cc1nc3ccccc3s1)CC2', 'O=C(/C=C/O)O[C@@H]1CCc2cc(OC(=O)[C@@H]3CCCCC[C@@H]3S(C)(=O)=O)ccc21', 'Clc1cccc2cccnc1n2-c1ccccc1', 'N=C1[C@H]2CCCCN1Cc1cc(F)ccc12', 'CN[C@H](N[C@@H]1CCOC1(C)C)c1nnc2n1CCCCC2', 'CNc1[nH+]/c(=C\\c2ccccc2Cl)nc([O-])c(OC)c1OC', 'O=[N+]([O-])c1c(F)cc2n1CN1CCCC[C@H]1CO2', 'O=C([O-])/C(C=C(/C#N)C1CCCCCC1)NNc1ccccc1C#CCO', 'O=C(C1CC=[NH+]C1)N1CCc2sccc2-c2cc(Cl)ccc21', 'O=C(/C=C\\C1C2CC(CC(C2)C1)[C@@H](CO)c1ccccc1O)N1CCc2ccccc21', 'O=C(C[NH+]1CCOc2ncccc2C1)Nc1ccc(Cl)c(F)c1', 'COc1nn[nH]c1CCc1nnc2n1CC[NH+](Cc1cccc(C(F)(F)F)c1)CC2', 'O=C(c1c[nH]c(-c2ccsc2)n1)N1CCC[NH2+]CC1', 'ClCc1n[nH]c2c(n1Cc1ccccc1)c(=O)n(CC(C(C)CO)[C@@H]2CNF)c1ccccc1', 'Clc1cc(Br)cc2ncnc(-c3cccs3)c1n2-c1ccccc1', 'Oc1cccc(O[C@@H]2[C@H](O)C(C(=O)[O-])CCS(=O)(=O)C2)c1', 'Oc1ccc([C@H]2C[NH+]=C(N3CCOCC3)Nc3ncnn32)cc1', 'O=C(/N=c1cc2oc3ccccc3c(=O)n1[C@@H]2c1cccs1)c1ccc(Cl)cc1', 'Clc1cccc(/N=N/OCC[NH+]2CCCCCC2)c1', 'Clc1cccc(F)c1C[NH+]1CCC[C@H]1C[C@@H]1CCC(=O)NC2=C1CCCC2', 'Clc1cccc(S[C@H](C[C@@H]2CCCCC[NH2+]2)c2cccs2)c1', 'Nc1n[nH]c(/N=C2\\C(=O)NC(=O)C([O-])=CC2=O)n1', 'OC[C@@H]1SC(N2CCCC2)=NC(=O)C[C@H]1[C@@H]2CCCN(C[C@H](O)c1ccccc1)CS2', 'OC[C@H]1O[C@@H](CO)[C@H](O)[C@H](O)[C@@H]2CCCC[C@H]12', 'Oc1ccccc1C12CC(CC(COc3ccccc3)CS2)[C@@H]1S', 'Clc1ccccc1-c1n[nH]c2c1[C@@H]1CC[C@H]1C[NH2+]C2', 'N[C@@H](Nc1nc2c(s1)C[NH+](C1CC1)CN(Cc1cccs1)C2)C1CC1', 'Oc1ccc([N+](=O)CCl)cc1OCC[NH+]1CCCCCC1', 'Nn1c(=O)oc(=O)n2ncn2c1=O', 'Clc1ccccc1Oc1cc(SC[C@@H]2CCOC3=c4ccccc4=[NH+][C@@H]3[C@H]2c2ccccc2)n(C2CC2)c1', 'OC[C@H]1[C@H]([NH2+]C2CCCCC2)CCCCC1(CCCC(F)(F)F)Nc1ccccc1', 'Nc1n[nH]c([C@@H]2CCCCC[C@@H]2[NH3+])n1', 'CNC(=O)[C@H]1CN[C@@H]2CCCN2C[C@@H]1C(C)C', 'NS(=O)(=O)N1CCC2(CC1)Oc1ccc(F)cc1C(c1ccccc1)(CCC[NH+]1CCOCC1)CO2', 'N=C1N2CCS(=O)(=O)C[C@@H]2[NH2+]Cc2ccccc21', 'Clc1ccccc1O/N=[N+]1/N=C/SCCS1', 'O[C@@H](C[C@@H]1CCOC2(CCCCCC2)C1)C(F)(F)F', 'Clc1ccc(Br)cc1[C@@H]1COCC[NH+](Cc2ccco2)C1', 'ClC[C@H]1CC[C@H](Cc2ccc(F)cc2)[N+]12CCCCCC2', 'Clc1ccccc1[C@H]1N=NC[C@H]2COCCS[C@@H]21', 'OCc1nnn2c1N[C@]1(C=CCC1)[C@H]2CC(=O)NC1CCCCCC1', 'Clc1cccc2[nH]nc(NCCC3=CCCOC3)n2n1', 'O=C(/N=c1\\[nH]c(S)nc2ccccc12)N1C[C@@H](O)[C@H]2C[C@@H]2[C@@H]2CCCC[C@H]21', 'Clc1nnnn1CCN1CCOCC[C@@H]1C[NH+]1CCCC1', 'O[C@@H](COCc1ccc2c(c1)CCCCC2)C[NH+]1CCC[C@@H](C(NC2CC2)C(F)(F)F)C1', 'C[C@@H](CNC[C@H]1CCCCC[NH2+]1)S(=O)(=O)c1nc(C2CCCCC2)no1', 'CN[C@H]1Nn2c(-c3ccc(F)cc3)cnc2N=C[C@@H]1SC', 'Nc1noc(C[NH+]2CCC[C@H](c3nnc(Nc4cccnc4)n3N)CN2)c1Cl', 'Clc1ccsc1N/C1=C2C=C[C@H](C2)c2ccccc21', 'Clc1ccccc1COCC(=O)N1C2CC3CC(C2)CC1C3', 'O[C@@H]1CCCCC2(CCN([NH2+]Cc3ccc(OC(F)F)c(F)c3)CC2)C1', 'Nc1[nH]nc2c1C[NH+](CC(=O)N(C1CC1)[C@H]1CCS(=O)(=O)C1)CCC2', 'Clc1cccc2cccnc1n2CC[NH2+]Cc1ccccc1', 'ClCc1noc([C@@H]2CCCCC[C@@H]2[NH2+][C@@H]2CCCC[C@@H]2O)n1', 'Clc1ccccc1N1N(C2CCCCCCC2)c2ccccc21', 'O=[N+]([O-])c1c(O)cc(NC[C@@H]2CCC[C@@H]2O)nc2n1CCCCC2', 'CCn1nc(C)cc1CN1C[C@H]2CC[C@@H]1CC(CC(F)F)C2', 'Oc1cccc2cc([C@@H]3CCC[NH+]3C3CC3)ccc(=O)c12', 'Clc1cccc([C@@H](N[C@@H]2CCc3nc[nH]c3C[NH+]2CC(F)(F)F)c2cccnc2)c1', 'Oc1ccccc1CC/[NH+]=N/CCC[NH+]1CCNN(C2CCOCC2)C(=O)C1', 'O=C(C[NH+]1CCC[NH+](Cc2cccc(Cl)c2)CC1)Nc1ccccc1F', 'N=C1N(C[C@H]([NH2+]C)c2ccccc2F)C(=O)NC([S-])=NC1', 'Cc1ncsc1C[C@H](CS)C[NH+]1CCCCCC1', 'O=C([O-])[C@H]1[C@@H]2c3cccc(F)c3CC[C@H]1O2', 'O=C(C[NH+]1CCC[C@](O)(C[NH+]2CCCCC2)C1)N[C@@H]1CCOc2ccccc2C1', 'O=C(CCc1cc(-c2ccccc2)n[nH]1)N1CCCCCCC1', 'N#C[C@@]1(C2CC[NH+](CC(=O)N[C@H]3CCOc4ccc(F)cc43)c3[nH]cnc3C2)CC1', 'Oc1ccc(F)c([C@@H]2O[C@@H]3CCC[C@H]2[C@@H](O)[C@@H]3O)c1', 'Oc1cc(Br)c([C@@H]2C[NH2+]CCC[C@H]2O)cc1F', 'Clc1ccsc1-c1cn(CCC(=O)N2CCCCCCC2)c2ccccc12', 'O=[N+]([O-])c1c(Br)sc2c1[C@H]1CC[C@@H]2C[C@@H](O)[C@@H]1N1c2ccccc2C1=O', 'O=C(c1c[nH]cn1)[C@H]1[C@@H](C2=CC=CC=SC2)N=NN1c1ccccc1Cl', 'O=[N+]([O-])c1cccc2c1[C@@H]1CCC[C@@H]2[C@@H]2C[C@H]([n+]3ccco3)CCC[C@@]21O', 'ClC(c1ccoc1C(F)(F)F)[C@@H]1C[C@@H](I)C=CC[C@H]1C(=O)[O-]', 'Clc1cc(N2CCC[C@@H](N3CC[NH+](CCO)C3)CC2)c[nH]1', 'O=[N+]([O-])c1c(-c2cc(=O)[nH]c3ccccc2c23CCOCC2)nccc1I', 'Clc1ccsc1[C@H]1n2cccc2C2=NC1=NC2', 'O[C@@H]1[C@@H]([NH2+]CC2CCCCC2)CSCCN1C[C@H](O)C(F)(F)F', 'COc1ncnc(N[C@H]2[C@@H]3CC[C@H](C2)[C@H]2C(=O)N[C@H]2C3C(=O)C(C)(C)C)c1O', 'O=[N+]([O-])c1ccnc2c1c(=O)n(Cc1cccnc1)[C@H](c1ccccc1Cl)c1ccccc1N2', 'N=C1C[C@]2(CCC[NH+](Cc3ccccc3)n2)Nc2cc(Cl)ccc2N1', 'Clc1ccsc1C[N-]/N=C/C[NH+]1CCCCCC1', 'O=C(COCCCOc1ccccc1)NCCC12CC3CC(CC(C3)C1)C2', 'OC1=C(Nc2nc([O-])c(-c3ccccc3)n3c4ccccc4n23)C2(CCC[NH2+]CC2)C1', 'Clc1ccccc1-c1nc(N2CCCCCCC2)co1', 'O=C(c1cn(Cc2cccnc2)c2c1CCCC2)N1CCCCCCC1', 'O=[N+]([O-])c1cc([C@H]2S[C@@H](O)C[C@H]3CC[C@H]2O3)cc(Br)c1O', 'OC1(Sc2ccccc2OCC2CCCCC2)CCCCCCC1', 'Oc1cc(F)nc(N2CCCCCCC2)c1', 'Clc1ccccc1NS[C@H]1[NH+](C2CC2)CCC[NH+](C2CC2)[C@@H]1C', 'O=C(CC(=O)N1CCOCCN(S(=O)(=O)c2ccc(Cl)cc2)C1)N1CCOCC1', 'Clc1ccccc1OCC1(C[NH2+]C2CC2)CCCCCCC1', 'OC1C[C@@H]2CC[C@@H]1C[NH+](Cc1ccccc1F)C2', 'Clc1ccc(C2SCCCCCC2)cc1F', 'Clc1cc(/N=C\\N2CCCCCCC2)ccc1[N+](=O)[O-]', 'COc1nc2sn(-c3ccccc3)c(=O)c2c2c1CCCCCC2', 'Oc1cccc2nnccc1c1ccccc12', 'Clc1cccc(/N=C2\\CCCCCc3nc[nH]c32)c1', 'Clc1ccsc1SC[C@H]1C[C@@]12OCCC[NH2+]C2', 'O=[N+]([O-])c1c(OCO)oc2c1COCCN(c1ccccc1)C2', 'ClC[C@H](C(=O)NCc1ccc([N+](=O)[O-])cc1)N1CCCSCCC1', 'Nc1c(Cl)c2c(cc1Cl)c1c([nH]c1=O)CCCC2', 'Clc1ccccc1Oc1cc(C[NH+]2CCC(Cc3nc4ccccc4[nH]3)CCCC2)ccc1O', 'O=C([C@H](CCO)Nc1cc(F)ccc1F)NC12CC3CC(CC(C3)C1)C2', 'O=C(Cc1cc2c(cc(Cl)cc1[N-]S(=O)(=O)c1ccccc1)n2Cc1ccccc1)NCc1ccccc1', 'Clc1ccccc1CNC12C[C@@H]3C[C@H](CC(C1(NC(=O)C3)C=C2)C(=O)[O-])c1ccccc1F', 'O=[N+]([O-])c1ccncc1[C@@H]1SCCCCc2ccccc21', 'O=C(c1cn[nH]c1-c1ccccc1)N1CC(C(=O)NCCC23CC4CC(CC(C4)C2)C3)C1', 'Oc1ccc([N+](=O)c2oc3c(c2[C@@H](NCc2ccccc2)C[C@H](c2ccc(F)cc2)N3)c2ccc(F)cc2)cn1', 'Clc1cc(Nc2nc3cc(Cl)ccn3n2)c2c(nc3ccccc13)CCC2', 'Clc1cccc2c(Oc1[O-])nc(Cn1cccn1)nn2C1CC1', 'Clc1cccc(CC[NH+]2CCCCCCC2)c1', 'NS(=O)(=O)Nc1ccc([N+](=O)[O-])c(NCC[NH+]2CCCCCCC2)c1', 'Clc1nnnc(SCn2ncc3ccc4c(c23)CCCCCC4)c1', 'N[C@@H]1[C@H](Cl)CCC[C@@H]([NH+]2C[C@@H](C)O[C@H](O)C2)C1', 'Clc1ccccc1NC12CC3CC(CC(C3)C1)C2', 'Clc1cccc(CC[NH+]2CCN([C@@H]3CCN(CC4(O)CCOCC4)CCCC3)CC2)c1', 'Oc1cccc(O)c1C(=O)N1CCC2(CC2)OCCN1Cc1ccccc1', 'O=C([C@H]1COc2cccc3cccc(c23)CN1C(=O)n1ncc(Br)c1Cl)N1CCCC1', 'Oc1cccc2cc(C(=O)Nc3ccc(F)cc3)cn-2/c([O-])c1', 'ClCc1csc2c1COCCCO2', 'CC[C@@H](CCO)NC(=O)N1C[C@@H](C)CCC[C@@H](C)C1', 'O=C(/N=C/NCCC[NH+]1CCCCCCC1)N[C@H]1CCCOc2ccccc21', 'O=[N+]([O-])c1ccnc2ccc([N-]S(=O)(=O)c3cc(F)ccc3F)cc1c1ccccc12', 'O=C(/N=c1\\[nH]c2c(ccc3ccccc13)[C@@H]1CCCN12)c1cccc(F)c1', 'NS(=O)(=O)/N=c1\\oc2c(cnc3ccccc13)C(=O)CCC2', 'ClCc1nnn2c1N[C@@H](c1ccccc1-n1nnnc1COCCO)C(=O)CCC2', 'Clc1ccccc1-c1nnc2n1NCC[C@@H]1C[C@@H]1C2', 'Clc1ccccc1O/C=Cc1ccc(N2S(=O)([O-])N=C(O)COc3ccccc32)cc1', 'Clc1nnnc(O[C@@H]2CCCCCC[C@@H]2O)n1', 'CNc1[nH+]ccc(C[NH+]2[C@@H](C)C[C@H]([NH2+]C)CC[C@@H]2C)n1', 'Clc1cc(Cl)c(N2CCN([C@@H]3[C@@H](O)NC2=O)CC3)cc1Cl', 'Oc1ccc([C@H]2CCCC(C#N)(C3CCOCC3)c3ccccc3N2)cc1', 'ClC[C@H]1C[NH+]2CCCC[C@H]1C2', 'Clc1cccc2c1O[C@@H](c1ccccc1Br)[C@@H]1C[C@@H]1[C@@H]1CCCN21', 'Clc1ccc(-c2nn([C@@H]3CCO[C@H]2c2ccccc2F)C3)cc1', 'Clc1ccc(CO)n1CC12CC3CC(CC(C3)C1)C2', 'Oc1ccccc1[C@H]1OP(=O)OCCCN1C(=O)c1ccccc1', 'Clc1cccc2[nH]nnc1c1ccccc12', 'Clc1ccccc1[C@H]([NH2+]C1CCCCC1)[C@H]1CO[C@@H]2C[C@@H]1c1ccccc1C2', 'Clc1ccccc1C1=CSCC[NH2+]CC1', 'O=C(COCc1ccncc1)NCCN1C[C@@H]2CC[C@@H]1CN(S(=O)(=O)c1cccs1)CC2', 'Clc1cccc(-c2cccc3c2O[C@@H](CN(C(C)C)C2CC3)C2)c1', 'Oc1cc(Br)c(Cl)cc1NCC[C@H]1CCCCCC[NH2+]1', 'Oc1cccc2nn[nH]c1c1ccccc12', 'N=C1[C@H]2CC3=CC=CC(C)(C)CC[C@@H]3C[C@@H](SCC(=O)Nc3ccccc3F)[C@@H]12', 'CCc1nc2n(n1)C=CC(C(=O)Nc1nc(C(C)(C)C)cs1)C(=O)C(C)(C)C2', 'O=[N+]([O-])c1cccc2[nH]cnc1c1cc(F)c(Cl)cc12', 'O=C(CC12CC(CC(CO)C1)CCC2)Nc1ccc(F)c(Cl)c1', 'Clc1cccc2[nH]nnc1c1nnc(NCc3ccccc3)n12', 'Clc1cccc2[nH]ncc1n1/C(=C/C(=O)N(Cc3ccccc3)c3ccccn3)c([O-])c12', 'O[C@@H]1CCCCCC[C@H]1[NH2+]C1CCCC1', 'Oc1ccccc1OC1CN(C(=O)c2cc(C3CC[NH+](C)C/C=C[C@H]3CO)[nH]n2)CC1', 'O[C@@H](c1csnn1)[C@H]1C[NH+]2CC[NH+]1CCC2', 'Nc1c(Cl)c([O-])c2cc(F)ccc2c1S(=O)(=O)/N=C(\\[O-])C12CC3CC(CC(C3)C1)C2', 'Clc1cccc2[nH]nnc1c1cc(C3CC3)nn12', 'CC12CCCC(CCCC2)CC1(Br)Br', 'OC[C@H](Br)CC12CC3CC(CC(C3)C1)C2', 'O[C@@H]1c2cc(C(F)(F)F)ccc2N[C@@H]2C[C@H]1C[C@H]2O', 'O=C(c1cc([S@](=O)C23CC4CC(CC(C4)C2)C3)oc1-n1cnnn1)c1ccccc1', 'Oc1cccc2cc([C@@H]3Nc4ncccc4C(=O)O3)c(O)cc1O[C@H]2c1cccc(Cl)c1', 'O=[N+]([O-])c1ccnc2cccnc1NC2', 'Clc1ccccc1OCc1csc(N2CCCCCCCC2)n1', 'O=C(COCCC12CC3CC(CC(C3)C1)[C@H]2C([O-])Cl)N1CCCCCC1', 'COC[C@](C)(C#N)NC(=O)CC12CC3C[C@](C)(C1)C[C@@](C)(C2)C3', 'Clc1ccccc1C1=C[C@@H]2[C@@H](CCCc3ccccc31)[C@@H]1CCC[C@@H]12', 'Clc1ccc(Br)c([C@H]2[C@H]3C=C4C[C@@H](CSC4)[C@H]3OC(=O)C[C@@H]2C(=O)[O-])c1', 'Oc1cccc(-n2c([O-])c3n(c2=O)CCCCCCC3)c1', 'Clc1cccc2ccc(Br)cc1CN2C[C@@H]1C[C@@H]2CC[C@@H]1CO2', 'Clc1ccc(C(=O)N[C@@H]2CCC(Cn3cc[nH+]c3)CCOCC2)cc1', 'Clc1cccc2c1nc(N1CC[C@@H](Oc3ccccc3)C1)OCCCO2', 'Cc1c(C(=O)N[C@H](C(=O)N2CCCC2)nncc1C(=O)N1CCCCC1)c1ccccc1', 'NS(=O)(=O)/C=c1/c(=N/C(=O)c2cc(Cc3ccccc3)n[nH]2)sc2c1CCCCCCC2', 'Clc1cc(Br)cc2c1OCCN1C[C@@H]1C[C@@H]2C(=O)[O-]', 'Nc1[nH]nc2c1CCCCC[NH+](Cc1ccccn1)C2', 'Oc1ccc(F)c(Sc2nc3c(cccc3c3C[NH+]4CCCCCC4)ccc23)c1', 'Cc1ccc(OCc2noc([C@H]3CCCN(C(=O)[C@H]4C[C@H]3C(F)(F)F)n4C)c2C)cc1C', 'Oc1ccc(Oc2cnc3c(c2)C[C@H](c2ccc(Br)s3)cc2)C1', 'O=[N+]([O-])c1c(Br)cccc1N1CCCCCCCCC1', 'ClCc1ccc(C2=CCN(Cc3ccccc3)CC1)CCCC2', 'O=[N+]([O-])c1c(F)cc2n1[C@@H](c1ccccc2F)c(C2CC1)CN2/N=C=C1/N=NCCS1', 'CN(CC[NH+](C)C)C(=O)c1ccc2c(c1)N(C)c1ncccc1N=C1C=CC=C1C=C2', 'O=C(CN(C[C@H](O)c1ccccc1)C1CCCOCC1)N1C[C@@H]2C[NH+](CCCNC1=O)C2=O', 'O=C(C1CC[NH+](C2CCCC2)[C@@H](C(=O)NC2CCCCCCCCC2)C1)c1ccccc1', 'Clc1ccc(CNc2ccc(C(=O)NNC(c3ccccc3)nc2)C2CC2)cc1', 'O=C([C@H]1[C@H]2C[C@@H](c3ccccc3O1)N(Cc1nc3ccccc3n12)C(=O)[O-])N1CCN(c2ccccc2)CC1', 'C[C@@H](NC(=O)CCl)[C@H]1CCc2c([nH]c3ccccc21)c1ccsc13', 'Clc1ccc(-c2n[nH]c(CN(Cc3ccco3)C2CCCCCCCOCCOCCO)c2)ccn12', 'Clc1cccc2[nH]nc(-c3nnc(-c4cccc(Cl)c4F)[nH]c13)CC2', 'Clc1ccccc1-c1nn(C[NH+]2CCN(C[C@@H](O)c3ccccc3)[C@@H](O)C1)C2', 'Clc1ccc(-c2n[nH]c(SCn3ccnn2)n3)cc1', 'O=C([O-])C1=NO[C@@H]2NC(=O)N(c3ccccc3)/C1=N/OCCC2', 'N#C[C@@]12CCC(CCNC(=O)c3sc4ncc(Cl)cc4c3[C@H]1C(=O)c1ccccc1)C2', 'Oc1cccc(C(=O)C2=CC(CCOc3ccc4c(c2)OCCO4)C3)c1', 'Clc1ccsc1NC(=S)Nc1ccc2c3c(cccc13)CC2', 'O=C(CCN[C@@H]1COc2ccccc21)Nc1ccc2c3c(cccc13)CS2', 'OCS[C@H]1CCCO[C@]2(CC[NH2+]C[C@H]2c2ccccc2)[C@@H]2OC(O)=C2CC1', 'Clc1ccsc1/N=C1/S[C@H](CC(=O)NC2=NN=CC3)CCCC3=C2C=C1', 'CC1=CC[C@]2(C)CCC[C@@H](C)C[C@H]1[NH2+]CC2', 'O=C([O-])[C@@H]1C[C@@H]2CC[P@]1[C@H]1O[C@@H]1[C@@H](c1ccc(Cl)cc1)[C@@H]1C[C@H]1O2', 'Clc1cccc2nn[nH]c1c1cccc2cc(Cl)c1F', 'Clc1ccc(-c2nnnn2/N=C2\\S[C@H]3OCCC[C@@H]13)CC(=O)N2Cc1ccccc1', 'Clc1ccc(Br)c([C@H]2[C@H]3CCCC(=O)N3C3CC[NH+](CCN4CCCCC4C2)C3)c1', 'Oc1cccc2cc(OC3CCC[C@]23CO)c[nH]1', 'Clc1nncn1Cc1ccc2c3c(cccc13)CCC2', 'Clc1cccc2[nH]ncc1n1c(C(=O)OCC(F)(F)F)c1C[NH+](Cc1ccccc1)C2', 'Oc1cc(-n2cn/nc3c4c(cccc24)OC[C@@H]3CO)cc1=O', 'Clc1cccc2[nH]nnc1c1nc2cc(Cl)ccc1OCC1CC1', 'ClCc1csc([C@H]2O[C@@H]3CCN(c4ccc(F)cc4)CCC[NH2+]CCC[C@@H](O)[C@H]32)n1', 'Clc1cccc2[nH]nnc1c1nc2ccccc1', 'OC1=CN=C2S[C@@H](COC3=c4ccccc4=[NH+][C@H]3C(=O)N2C2CC1)SCCO2', 'Oc1cccc2ncn(/c3c(C(=O)NCCc4ccccc4)c3ccccc2)c1', 'O=[N+]([O-])c1cccc2[nH]cn/c(=N/n3c4ccccc4nc3c1)OCOC2', 'Clc1cccc2ncn(C3CC[NH+]=C3C(=O)NCCN2c2ccccc2Cl)c1', 'Clc1ccc(CC2=NC(=O)/C3=NC4=CC(NC(=O)CS/C4=NC/C(=C/c4ccccn4)C3)C2)cc1OCCO', 'Clc1cccc2[nH]nc(N/N=C/Cn1)CCCC2=O', 'Oc1cccc2nc(SCCSc3nc4ccccc4cc3n1)CCC2']

# 
ten_mob = sorted(All_mobility_list, reverse=True)[:10]
print('mobility_list_10000_2', ten_mob)  
print(sum(ten_mob)/len(ten_mob))

# スコアの詳細
ten = sorted(All_score_list, reverse=True)[:10]
print('score_list_10000_2', ten)  
print(sum(ten)/len(ten))
